## **Apache Spark and PySpark Installation**

In [ ]:
#'''
# Install spark-related dependencies

#install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

#download spark3.0.1
!wget -q http://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz

#unzip it
!tar xf spark-3.0.1-bin-hadoop2.7.tgz

#install findspark
!pip install -q findspark
!pip install pyspark
#'''

In [ ]:
#'''
# Set up required environment variables

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"
#'''

In [ ]:
#'''
print(os.listdir('./sample_data'))
#'''

['README.md', 'anscombe.json', 'california_housing_test.csv', 'mnist_test.csv', 'mnist_train_small.csv', 'california_housing_train.csv']


In [ ]:
#'''
file_loc = './sample_data/california_housing_train.csv'
#'''

In [ ]:
#'''
#Run a local spark session to test installation

import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

# Test the spark
df = spark.read.csv(file_loc, inferSchema=True, header = True)
print(type(df))
df.show(3, False)
#'''

<class 'pyspark.sql.dataframe.DataFrame'>
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|-114.31  |34.19   |15.0              |5612.0     |1283.0        |1015.0    |472.0     |1.4936       |66900.0           |
|-114.47  |34.4    |19.0              |7650.0     |1901.0        |1129.0    |463.0     |1.82         |80100.0           |
|-114.56  |33.69   |17.0              |720.0      |174.0         |333.0     |117.0     |1.6509       |85700.0           |
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
only showing top 3 rows



In [ ]:
# Point Colaboratory to your Google Drive

from google.colab import drive
#drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


### **Import Necessary Libraries**

In [ ]:
import librosa 
import librosa.display
!pip install SoundFile
import soundfile 
import os, glob
from time import time
import pickle 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.neural_network import MLPClassifier 
from sklearn.svm import SVC 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import make_scorer, confusion_matrix, classification_report

## **Feature Extraction**

In [ ]:
def extract_feature(file_name, mfcc, chroma, mel, contrast, tonnetz):
    with soundfile.SoundFile(file_name) as sound_file:
        X,sr = librosa.core.load(file_name, mono=True, dtype="float32")
        #X = sound_file.read(always_2d=True, dtype="float32")
        sample_rate=sound_file.samplerate
        #sound_file.channels = 1
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
        if contrast:
            contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, contrast))
        if tonnetz:
            tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
            result = np.hstack((result, tonnetz))
    return result

In [ ]:
#emotions as suggested in the RAVDESS dataset filenames
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#classes we want to classify the emotions in
observed_emotions=['03', '04', '05', '08']

## **Load Dataset**

In [ ]:
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("/content/gdrive/My Drive/Colab Datasets/RAVDESS_Dataset/Actor_*//*.wav"):
        file_name=os.path.basename(file)
        emotion=file_name.split("-")[2]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True, contrast=True, tonnetz=True)
        x.append(feature)
        y.append(emotion)
        #tvs = TrainValidationSplit(trainRatio=0.5)
#    return tvs
    return x,y


#import IPython.display as ipd
#import matplotlib.pyplot as plt

#ipd.Audio('/content/gdrive/My Drive/Colab Datasets/RAVDESS_Dataset/Actor_01/03-01-01-01-01-01-01.wav')

#data, sampling_rate = librosa.load('/content/gdrive/My Drive/Colab Datasets/RAVDESS_Dataset/Actor_01/03-01-01-01-01-01-01.wav')
#plt.figure(figsize=(12, 4))
#librosa.display.waveplot(data, sr=sampling_rate)


## **Split Dataset**

In [ ]:
x,y=load_data(test_size=0.25)

In [ ]:
z = train_test_split(np.array(x), y, test_size=0.25, random_state=9)
x_train, x_test, y_train, y_test = z

In [ ]:
print("No. of training samples:", x_train.shape[0]," No. of testing samples:", x_test.shape[0])
#print(type(x_train))
x=np.array(x)
print(x.shape)
#print(type(x))
#print(type(y))
#print(type(y_train))

No. of training samples: 504  No. of testing samples: 168
(672, 193)


## **Feature Extraction**

In [ ]:
print(f'No. of Features extracted: {x_train.shape[1]}')

No. of Features extracted: 193


In [ ]:
'''
!pip install pydub
from pydub import AudioSegment
for file in glob.glob("/content/gdrive/My Drive/Colab Datasets/RAVDESS_Dataset/Actor_*//*.wav"):
  wav_file = AudioSegment.from_file(file = file, format="wav")
  #print(wav_file.channels)
  wav_file = wav_file.set_channels(1)

  wav_file.export(path.join(content/gdrive/My Drive/Colab Datasets/Dataset), "file.wav", format="wav")
'''

'\n!pip install pydub\nfrom pydub import AudioSegment\nfor file in glob.glob("/content/gdrive/My Drive/Colab Datasets/RAVDESS_Dataset/Actor_*//*.wav"):\n  wav_file = AudioSegment.from_file(file = file, format="wav")\n  #print(wav_file.channels)\n  wav_file = wav_file.set_channels(1)\n\n  wav_file.export(path.join(content/gdrive/My Drive/Colab Datasets/Dataset), "file.wav", format="wav")\n'

##**Feature Selection**

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
s = StandardScaler()
s.fit(x_train)
#print(X_train)
x_train_scaled = s.transform(x_train)
x_test_scaled = s.transform(x_test)

#Perform PCA

pca = PCA()
x_train = pca.fit_transform(x_train_scaled)
x_test = pca.transform(x_test_scaled)

explained_variance = pca.explained_variance_ratio_
print("explained_variance ",explained_variance)

explained_variance  [2.57389090e-01 8.47241531e-02 5.26199794e-02 4.81547657e-02
 4.06468677e-02 3.24882054e-02 2.97072195e-02 2.75447289e-02
 2.54223608e-02 2.43796607e-02 2.07316568e-02 1.85745279e-02
 1.70296119e-02 1.64438094e-02 1.52339500e-02 1.34068550e-02
 1.20916937e-02 1.17783281e-02 1.13203735e-02 1.09646602e-02
 9.84151914e-03 9.64557740e-03 9.29649416e-03 8.64190589e-03
 8.34043009e-03 7.47071412e-03 7.32986401e-03 7.03032355e-03
 6.75405778e-03 6.25675978e-03 5.91094808e-03 5.71195334e-03
 5.28144432e-03 5.02442493e-03 4.82750334e-03 4.48418830e-03
 4.41535773e-03 4.26100166e-03 4.20828876e-03 3.90664412e-03
 3.72084979e-03 3.67124312e-03 3.39986735e-03 3.35297916e-03
 3.26704909e-03 3.12117314e-03 2.93796083e-03 2.84675284e-03
 2.74318832e-03 2.64450261e-03 2.49915188e-03 2.40538139e-03
 2.30968367e-03 2.21805823e-03 2.11281936e-03 2.08987721e-03
 2.02546724e-03 2.00266328e-03 1.91190332e-03 1.84987505e-03
 1.76765884e-03 1.71352784e-03 1.63462056e-03 1.54060951e-03
 1.4

# **Classification - sklearn**

In [ ]:
scoring = {'accuracy':make_scorer(accuracy_score),
           'precision':make_scorer(precision_score, average='micro'),
           'recall':make_scorer(recall_score, average='micro'),
           'f1_score':make_scorer(f1_score, average='micro')}

In [ ]:
#Initialize all the ml classifiers

#Multilayer Perceptron Classifier
m1_model=MLPClassifier(alpha=0.005, batch_size=256, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

#Support Vector Classifier
s1_model=SVC(C=0.001, kernel='poly', gamma=0.001)

#Random Forest Classifier
r1_model=RandomForestClassifier(max_depth=7, max_features=0.5, min_samples_leaf=1, min_samples_split=2, n_estimators=40)

#Decision Tree Classifier
d1_model=DecisionTreeClassifier(criterion='entropy', max_depth=7, max_features=None, min_samples_leaf=1, min_samples_split=2)

In [ ]:
'''
#Classifier evaluation function

def classifier_evaluation(X, y, folds):

  X : features
  y : target
  folds : no. of cross-validation folds
  
  
  m1 = cross_validate(m1_model, X, y, cv=folds, scoring=scoring)
  s1 = cross_validate(s1_model, X, y, cv=folds, scoring=scoring)
  r1 = cross_validate(r1_model, X, y, cv=folds, scoring=scoring)
  d1 = cross_validate(d1_model, X, y, cv=folds, scoring=scoring)

  #Data frame for score table

  classifier_score_table = pd.DataFrame({'Multilayer Perceptron Classifier':[m1['test_accuracy'].mean(),
                                                                             m1['test_precision'].mean(),
                                                                             m1['test_recall'].mean(),
                                                                             m1['test_f1_score'].mean()],
                                         
                                         'Support Vector Classifier':[s1['test_accuracy'].mean(),
                                                                      s1['test_precision'].mean(),
                                                                      s1['test_recall'].mean(),
                                                                      s1['test_f1_score'].mean()],
                                         
                                         'Random Forest Classifier':[r1['test_accuracy'].mean(),
                                                                     r1['test_precision'].mean(),
                                                                     r1['test_recall'].mean(),
                                                                     r1['test_f1_score'].mean()],
                                         
                                         'Decision Tree Classifier':[d1['test_accuracy'].mean(),
                                                                     d1['test_precision'].mean(),
                                                                     d1['test_recall'].mean(),
                                                                     d1['test_f1_score'].mean()]},
                                        
                                          index=['Accuracy', 'Precision', 'Recall', 'F1 Score'])

  #Best Score Column
  classifier_score_table['Best Score']=classifier_score_table.idxmax(axis=1)

  return(classifier_score_table)

classifier_evaluation(x_train, y_train, 5)
'''

"\n#Classifier evaluation function\n\ndef classifier_evaluation(X, y, folds):\n\n  X : features\n  y : target\n  folds : no. of cross-validation folds\n  \n  \n  m1 = cross_validate(m1_model, X, y, cv=folds, scoring=scoring)\n  s1 = cross_validate(s1_model, X, y, cv=folds, scoring=scoring)\n  r1 = cross_validate(r1_model, X, y, cv=folds, scoring=scoring)\n  d1 = cross_validate(d1_model, X, y, cv=folds, scoring=scoring)\n\n  #Data frame for score table\n\n  classifier_score_table = pd.DataFrame({'Multilayer Perceptron Classifier':[m1['test_accuracy'].mean(),\n                                                                             m1['test_precision'].mean(),\n                                                                             m1['test_recall'].mean(),\n                                                                             m1['test_f1_score'].mean()],\n                                         \n                                         'Support Vector Classifier':[s1['

### **Multi Layer Perceptron**

In [ ]:

#initialize the model
m1=MLPClassifier(alpha=0.005, batch_size=256, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

#Train the model
m1.fit(x_train,y_train)

#Predict for the test set
m1y_pred=m1.predict(x_test)

#Calculate the accuracy of our model
acc_m1=accuracy_score(y_true=y_test, y_pred=m1y_pred)
#f_s1=fbeta_score(y_test, s1y_pred, beta=0.5, average='weighted')
f1_m1=f1_score(y_true=y_test, y_pred=m1y_pred, average='weighted')
wP_m1=precision_score(y_true=y_test, y_pred=m1y_pred, average='weighted')
wR_m1=recall_score(y_true=y_test, y_pred=m1y_pred, average='weighted')
'''
#Calculate fbeta score
f_m1=fbeta_score(y_test, m1y_pred, beta=0.5, average='micro')
'''
#Calculate confusion matrix
#cm1=confusion_matrix(y_test, m1y_pred, labels=observed_emotions, average='weighted').astype(np.float32)

#Print the accuracy
print("Accuracy for MLP : {:.2f}%".format(acc_m1*100))
print("d1 for MLP : {:.2f}%".format(f1_m1*100))
print("wp for MLP : {:.2f}%".format(wP_m1*100))
print("wr for MLP : {:.2f}%".format(wR_m1*100))
'''
#Display confusion matrix
plt.imshow(cm1, cmap="binary")
plt.show()
'''

Accuracy for MLP : 67.26%
d1 for MLP : 67.36%
wp for MLP : 68.93%
wr for MLP : 67.26%


'\n#Display confusion matrix\nplt.imshow(cm1, cmap="binary")\nplt.show()\n'

### **Support Vector Machine**

In [ ]:
'''
#initialize the model
s1=SVC(C=0.001, kernel='poly', gamma=0.001)

#Train the model
s1.fit(x_train, y_train)

#Predict for the test set
s1y_pred=s1.predict(x_test)

#Calculate the accuracy of our model
acc_s1=accuracy_score(y_true=y_test, y_pred=s1y_pred)

#Calculate fbeta score
f_s1=fbeta_score(y_test, s1y_pred, beta=0.5, average='micro')

#Calculate confusion matrix
cm2=confusion_matrix(y_test, s1y_pred, labels=observed_emotions).astype(np.float32)

#Print the accuracy
print("Accuracy for SVC : {:.2f}%".format(acc_s1*100))

#Display confusion matrix
plt.imshow(cm2, cmap="binary")
plt.show()
'''

'\n#initialize the model\ns1=SVC(C=0.001, kernel=\'poly\', gamma=0.001)\n\n#Train the model\ns1.fit(x_train, y_train)\n\n#Predict for the test set\ns1y_pred=s1.predict(x_test)\n\n#Calculate the accuracy of our model\nacc_s1=accuracy_score(y_true=y_test, y_pred=s1y_pred)\n\n#Calculate fbeta score\nf_s1=fbeta_score(y_test, s1y_pred, beta=0.5, average=\'micro\')\n\n#Calculate confusion matrix\ncm2=confusion_matrix(y_test, s1y_pred, labels=observed_emotions).astype(np.float32)\n\n#Print the accuracy\nprint("Accuracy for SVC : {:.2f}%".format(acc_s1*100))\n\n#Display confusion matrix\nplt.imshow(cm2, cmap="binary")\nplt.show()\n'

### **Random Forest Classifier**

In [ ]:

#initialize the model
r1=RandomForestClassifier(max_depth=7, max_features=0.5, min_samples_leaf=1, min_samples_split=2, n_estimators=40)

#Train the model
r1.fit(x_train, y_train)

#Predict for the test set
r1y_pred=r1.predict(x_test)

#Calculate the accuracy of our model

acc_r1=accuracy_score(y_true=y_test, y_pred=r1y_pred)
#f_s1=fbeta_score(y_test, s1y_pred, beta=0.5, average='weighted')
f1_r1=f1_score(y_true=y_test, y_pred=r1y_pred, average='weighted')
wP_r1=precision_score(y_true=y_test, y_pred=r1y_pred, average='weighted')
wR_r1=recall_score(y_true=y_test, y_pred=r1y_pred, average='weighted')
#Calculate fbeta score
#f_r1=fbeta_score(y_test, r1y_pred, beta=0.5, average='micro')

#Calculate confusion matrix
#cm3=confusion_matrix(y_test, r1y_pred, labels=observed_emotions).astype(np.float32)

#Print the accuracy
print("Accuracy for MLP : {:.2f}%".format(acc_r1*100))
print("d1 for MLP : {:.2f}%".format(f1_r1*100))
print("wp for MLP : {:.2f}%".format(wP_r1*100))
print("wr for MLP : {:.2f}%".format(wR_r1*100))
'''
plt.imshow(cm3, cmap="binary")
plt.show()
'''

Accuracy for MLP : 57.14%
d1 for MLP : 57.12%
wp for MLP : 57.16%
wr for MLP : 57.14%


'\nplt.imshow(cm3, cmap="binary")\nplt.show()\n'

### **Decision Tree Classifier**

In [ ]:

#initialize the model
d1=DecisionTreeClassifier(criterion='entropy', max_depth=7, max_features=None, min_samples_leaf=1, min_samples_split=2)

#Train the model
d1.fit(x_train, y_train)

#Predict for the test set
d1y_pred=d1.predict(x_test)


acc_d1=accuracy_score(y_true=y_test, y_pred=d1y_pred)
#f_s1=fbeta_score(y_test, s1y_pred, beta=0.5, average='weighted')
f1_d1=f1_score(y_true=y_test, y_pred=d1y_pred, average='weighted')
wP_d1=precision_score(y_true=y_test, y_pred=d1y_pred, average='weighted')
wR_d1=recall_score(y_true=y_test, y_pred=d1y_pred, average='weighted')
#Calculate fbeta score
#f_r1=fbeta_score(y_test, r1y_pred, beta=0.5, average='micro')

#Calculate confusion matrix
#cm3=confusion_matrix(y_test, r1y_pred, labels=observed_emotions).astype(np.float32)

#Print the accuracy
print("Accuracy for MLP : {:.2f}%".format(acc_d1*100))
print("d1 for MLP : {:.2f}%".format(f1_d1*100))
print("wp for MLP : {:.2f}%".format(wP_d1*100))
print("wr for MLP : {:.2f}%".format(wR_d1*100))

'''
#Display confusion matrix
plt.imshow(cm4, cmap="binary")
plt.show()
'''

Accuracy for MLP : 47.62%
d1 for MLP : 47.31%
wp for MLP : 47.30%
wr for MLP : 47.62%


'\n#Display confusion matrix\nplt.imshow(cm4, cmap="binary")\nplt.show()\n'

# **Classification - PySpark**

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.tuning import TrainValidationSplit
#from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
#np to spark Dataframe and Train Test dataset
newy = np.array(y)
newy = newy.reshape(672,1)
df = np.concatenate([x, newy],axis=1)
print(df)
print(df.shape)

dff = map(lambda x: (Vectors.dense(x[0:193]) , (int(x[193]))), df)

mydf = spark.createDataFrame(dff,schema=["features", "final"])
indexer = StringIndexer(inputCol="final", outputCol="label")
mydf = indexer.fit(mydf).transform(mydf)
mydf.show()
print(type(mydf))
print(mydf.show(5,False))
print(mydf.printSchema())
splits=mydf.randomSplit([0.7, 0.3],seed=None)
train = splits[0]
test = splits[1]


[['-777.866157611455' '56.877395230111105' '-10.60628540017166' ...
  '0.008244789217204866' '-0.005095628009326177' '01']
 ['-440.3773090262629' '-13.673679986042893' '-38.65689286341312' ...
  '-0.002579203364031299' '0.005030455450146636' '05']
 ['-570.8517482000967' '29.91189170443348' '-25.305945501818982' ...
  '-0.007345664271454203' '0.0014896882305137009' '05']
 ...
 ['-616.300328563892' '24.223742046184665' '-21.477314108579023' ...
  '0.0032347977041022077' '0.0030059822982700943' '04']
 ['-715.9798129314961' '42.33756341480567' '-12.061107712588683' ...
  '-0.006744289765745544' '0.0030138744951017206' '04']
 ['-635.4520712650874' '35.57470759946204' '-26.71050384638229' ...
  '0.002893100368124395' '0.009884228425074823' '03']]
(672, 194)
+--------------------+-----+-----+
|            features|final|label|
+--------------------+-----+-----+
|[-777.86615761145...|    1|  3.0|
|[-440.37730902626...|    5|  2.0|
|[-570.85174820009...|    5|  2.0|
|[-427.66609430635...|    5|

In [ ]:
#DO NOT RUN THIS
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from pyspark.ml.feature import PCA as PCAml


mydf.show()
print(type(mydf))
print(mydf.show(5,False))
print(mydf.printSchema())
splits = mydf.randomSplit([0.7, 0.3], seed=None)
train = splits[0]
test = splits[1]

pca = PCAml(inputCol="features",outputCol="finalfeatures")
model = pca.fit()
transformed = model.transform(mydf)
print(transformed)
'''



s = StandardScaler()
s.fit(train)
#print(train)
train_scaled = s.transform(train)
test_scaled = s.transform(test)

#Perform PCA

pca = PCA()
train = pca.fit_transform(train_scaled)
test = pca.transform(test_scaled)

explained_variance = pca.explained_variance_ratio_
print("explained_variance ",explained_variance)
'''

+--------------------+-----+-----+
|            features|final|label|
+--------------------+-----+-----+
|[-777.86615761145...|    1|  3.0|
|[-440.37730902626...|    5|  2.0|
|[-570.85174820009...|    5|  2.0|
|[-427.66609430635...|    5|  2.0|
|[-418.18285878910...|    5|  2.0|
|[-652.67270988089...|    3|  0.0|
|[-432.58834752868...|    5|  2.0|
|[-512.77494840313...|    4|  1.0|
|[-771.57374557006...|    1|  3.0|
|[-761.76522695479...|    1|  3.0|
|[-529.90449413877...|    4|  1.0|
|[-669.78263202555...|    4|  1.0|
|[-457.64756461535...|    3|  0.0|
|[-718.47102243127...|    4|  1.0|
|[-585.06064134358...|    5|  2.0|
|[-465.40956172455...|    3|  0.0|
|[-685.03095402127...|    4|  1.0|
|[-542.36444250237...|    4|  1.0|
|[-700.29968683984...|    4|  1.0|
|[-652.38740726542...|    3|  0.0|
+--------------------+-----+-----+
only showing top 20 rows

<class 'pyspark.sql.dataframe.DataFrame'>
+-------------------------------------------------------------------------------------------

TypeError: ignored

### **Multi Layer Perceptron**

In [ ]:
#initialize the model
layers = [193, 40, 80, 40, 4]
mlpc = MultilayerPerceptronClassifier(layers = layers, maxIter=500, blockSize = 256, seed = 4, solver='gd', labelCol='label',featuresCol='features')

#Training and Transforming the model
models = mlpc.fit(train)
result = models.transform(test)

#Testing the model
predictionAndLabels = result.select("prediction", "label")

#Calculating the accuracy of the model
evaluator_acc_m2 = MulticlassClassificationEvaluator(metricName="accuracy")
evaluator_wR_m2 = MulticlassClassificationEvaluator(metricName="weightedRecall")
evaluator_wP_m2 = MulticlassClassificationEvaluator(metricName="weightedPrecision")
evaluator_f1_m2= MulticlassClassificationEvaluator(metricName="f1")

acc_m2=evaluator_acc_m2.evaluate(predictionAndLabels)
f1_m2=evaluator_f1_m2.evaluate(predictionAndLabels)
wP_m2=evaluator_wP_m2.evaluate(predictionAndLabels)
wR_m2=evaluator_wR_m2.evaluate(predictionAndLabels)
#Printing the accuracy
print("Accuracy for MLP : {:.2f}%".format(acc_m2*100))
print("F1 Score for MLP : {:.2f}%".format(f1_m2*100))
print("wP for MLP : {:.2f}%".format(wP_m2*100))
print("wR for MLP : {:.2f}%".format(wR_m2*100))


Accuracy for MLP : 41.03%
F1 Score for MLP : 29.93%
wP for MLP : 23.56%
wR for MLP : 41.03%


### **Random Forest Classifier**

In [ ]:
# Train a RandomForest model.
model1 = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=10)

# Train model.  This also runs the indexers.
r2 = model1.fit(train)

# Make predictions.
predictions = r2.transform(test)

# Select example rows to display.
predictions.select("prediction", "label", "features").show(5)

# Select (prediction, true label) and compute test error

evaluator_acc_r2 = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="accuracy")
evaluator_wR_r2 = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="weightedRecall")
evaluator_wP_r2 = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="weightedPrecision")
evaluator_f1_r2= MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="f1")
#Calculating the accuracy of the model
acc_r2=evaluator_acc_r2.evaluate(predictions)
f1_r2=evaluator_f1_r2.evaluate(predictions)
wP_r2=evaluator_wP_r2.evaluate(predictions)
wR_r2=evaluator_wR_r2.evaluate(predictions)
#Printing the accuracy
print("Accuracy for Random Forest Classifier : {:.2f}%".format(acc_r2*100))
print("F1 Score for Random Forest Classifier : {:.2f}%".format(f1_r2*100))
print("wP for Random Forest Classifier : {:.2f}%".format(wP_r2*100))
print("wR for Random Forest Classifier : {:.2f}%".format(wR_r2*100))

#print(r2.toDebugString)
#rfModel = model.stages
#print(rfModel)  # summary only

### **Decision Tree Classifier**

In [ ]:
model2 = DecisionTreeClassifier(labelCol="label", featuresCol="features")
d2 = model2.fit(train)

# Make predictions.
predictions = d2.transform(test)

# Select example rows to display.
predictions.select("prediction", "label", "features").show(5)

# Select (prediction, true label) and compute accuracy
evaluator_acc_d2 = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="accuracy")
evaluator_wR_d2 = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="weightedRecall")
evaluator_wP_d2 = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="weightedPrecision")
evaluator_f1_d2= MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="f1")
#Calculating the accuracy of the model
acc_d2=evaluator_acc_d2.evaluate(predictions)
f1_d2=evaluator_f1_d2.evaluate(predictions)
wP_d2=evaluator_wP_d2.evaluate(predictions)
wR_d2=evaluator_wR_d2.evaluate(predictions)
#Printing the accuracy
print("Accuracy for Decision Tree Classifier : {:.2f}%".format(acc_d2*100))
print("F1 Score for Decision Tree Classifier : {:.2f}%".format(f1_d2*100))
print("wP for Decision Tree Classifier : {:.2f}%".format(wP_d2*100))
print("wR for Decision Tree Classifier : {:.2f}%".format(wR_d2*100))

treeModel = d2.toDebugString
# summary only
#print(treeModel)

In [ ]:
'''
count=0
for i in x:
  print("Row(",end =" ")
  for j in range(len(i)-1):
    print("P"+str(count)+"="+str(i[j])+",",end =" ")
    count+=1
  print("P"+str(count)+"="+str(i[len(i)-1])+"),")
  count=0
#temp_df = spark.createDataFrame([Row(V4366=0.0, V4460=0.232, V4916=-0.017, V1495=-0.104, V1639=0.005, V1967=-0.008, V3049=0.177, V3746=-0.675, V3869=-3.451, V524=0.004, V5409=0), Row(V4366=0.0, V4460=0.111, V4916=-0.003, V1495=-0.137, V1639=0.001, V1967=-0.01, V3049=0.01, V3746=-0.867, V3869=-2.759, V524=0.0, V5409=0), Row(V4366=0.0, V4460=-0.391, V4916=-0.003, V1495=-0.155, V1639=-0.006, V1967=-0.019, V3049=-0.706, V3746=0.166, V3869=0.189, V524=0.001, V5409=0), Row(V4366=0.0, V4460=0.098, V4916=-0.012, V1495=-0.108, V1639=0.005, V1967=-0.002, V3049=0.033, V3746=-0.787, V3869=-0.926, V524=0.002, V5409=0), Row(V4366=0.0, V4460=0.026, V4916=-0.004, V1495=-0.139, V1639=0.003, V1967=-0.006, V3049=-0.045, V3746=-0.208, V3869=-0.782, V524=0.001, V5409=0)])
'''

In [ ]:
'''temp_df = spark.createDataFrame([Row( P0=-710.7831744230725, P1=45.55715047625142, P2=-7.513659507987078, P3=-3.6859329724151304, P4=-14.895490251810237, P5=-10.811491449744993, P6=-7.077374855894155, P7=-8.89312239105648, P8=-1.1166360851314572, P9=-11.659425231920086, P10=-10.138837391497319, P11=-8.41960189019282, P12=-8.451468287217809, P13=-7.661596117515486, P14=-3.4950267307533096, P15=-0.33083262663599944, P16=-1.0130057301039725, P17=4.8234378174728425, P18=7.675489974934347, P19=6.03986071617615, P20=4.604653395523952, P21=3.9286603714264863, P22=1.4807035256940606, P23=2.1280394189618135, P24=1.9227043954663756, P25=1.8062396809960894, P26=0.06288506230831027, P27=-1.514642944977314, P28=-4.243787282839509, P29=-0.7789525893165837, P30=-1.7330052835379453, P31=-0.31316063150348866, P32=-0.9166394197895422, P33=-0.8303310884128308, P34=-1.7074754978660398, P35=1.0653410772032297, P36=-2.3316465252818337, P37=1.1262265276942602, P38=-2.859635108593634, P39=-0.2544554742554502, P40=0.6625723049416973, P41=0.6098844287555452, P42=0.6029355464708143, P43=0.6152376341245674, P44=0.6342394147037768, P45=0.6660812839915917, P46=0.6363419018697309, P47=0.6260192117124871, P48=0.636187563090112, P49=0.6516499943598817, P50=0.7150688421571036, P51=0.7061978175694981, P52=9.961596639603573e-07, P53=2.445901069427373e-06, P54=7.649158074761857e-06, P55=1.8965901187045902e-05, P56=3.094139095171672e-05, P57=3.6067752049861674e-05, P58=1.8616133223395914e-05, P59=2.701122128021617e-05, P60=0.0002536696123545847, P61=0.004262228506245559, P62=0.014900253834113248, P63=0.02077846144970561, P64=0.02168628474311635, P65=0.07034764997637911, P66=0.09468627421907816, P67=0.04710031603179809, P68=0.008528464253547962, P69=0.0006276328151118109, P70=0.0002536044636670287, P71=0.0002591536652102435, P72=0.00134903298075833, P73=0.0016690106634434647, P74=0.0034876574445391866, P75=0.006762157978367063, P76=0.008827430923822112, P77=0.009158220946559022, P78=0.003970642577857511, P79=0.005443013558426565, P80=0.014133010130656374, P81=0.009438834734248915, P82=0.010573909146478766, P83=0.011611009313359903, P84=0.0040371879561759095, P85=0.0024144657764305563, P86=0.0030742966268612784, P87=0.004635484876555145, P88=0.008843822707644201, P89=0.0074388736774211595, P90=0.0026686716002122077, P91=0.002489184177755088, P92=0.002811355816155648, P93=0.0026277612532508124, P94=0.0025305313755509856, P95=0.0017026267329332452, P96=0.00040495270537865293, P97=0.000545296935291741, P98=0.0005199946001096201, P99=0.00038071453455792103, P100=0.00024253730574983002, P101=0.0011111203953312263, P102=0.0012949147722704898, P103=0.00015959279106276953, P104=0.00012896970932359941, P105=0.00012341395349396215, P106=8.360953501829123e-05, P107=5.058218037167512e-05, P108=0.0009382547586771937, P109=0.000890276895187976, P110=0.00017747068870379903, P111=6.970301559734928e-05, P112=4.2734796189021066e-05, P113=0.00023617606161816402, P114=0.0010937618148985882, P115=0.0003226244556988901, P116=4.3832556983774115e-05, P117=5.004344228372786e-05, P118=5.113608901808481e-05, P119=9.251156788525813e-05, P120=9.905570023532543e-05, P121=8.99304830904925e-05, P122=6.0780960266867085e-05, P123=5.236339634375866e-05, P124=5.743674042357759e-05, P125=4.099987772585881e-05, P126=2.380520052878075e-05, P127=4.3152447127610364e-05, P128=2.6971747236688974e-05, P129=1.3512404310639468e-05, P130=7.281922948103569e-06, P131=5.01257711630927e-06, P132=1.0144607649162871e-05, P133=3.7784180709178777e-06, P134=4.593510640706466e-06, P135=7.962006671281153e-06, P136=6.727584612292818e-06, P137=1.4076774677854338e-05, P138=1.6784088087885124e-05, P139=3.624105947490277e-05, P140=3.935371145618965e-05, P141=2.82640486759242e-05, P142=2.3382507659608487e-05, P143=1.716210163787645e-05, P144=1.0646239595300081e-05, P145=7.943973365791695e-06, P146=1.0322451783854788e-05, P147=1.178710837789944e-05, P148=1.9978721810389326e-05, P149=1.8124192551877097e-05, P150=1.8260563668157375e-05, P151=9.906963431490915e-06, P152=6.59498437528006e-06, P153=6.8637827449852555e-06, P154=1.666795815803245e-05, P155=1.3751282871964458e-05, P156=1.7083525267836865e-05, P157=8.760726543314834e-06, P158=9.372476980827446e-06, P159=5.428681421323017e-06, P160=5.931301616862024e-06, P161=5.825242349883081e-06, P162=8.455156355648875e-06, P163=1.1452697585287235e-05, P164=7.898817222409344e-06, P165=7.88366443098465e-06, P166=4.906532320130397e-06, P167=2.0995641366366716e-06, P168=3.207841020473656e-06, P169=4.940413301710799e-06, P170=7.1761591489765795e-06, P171=1.0816591395920958e-05, P172=7.991706984325373e-06, P173=6.477154058520059e-06, P174=8.950071959442502e-06, P175=1.4676425363502884e-05, P176=1.8862827984754974e-05, P177=7.638064008322882e-05, P178=3.249674482734368e-05, P179=1.1169616266360203e-06),Row( P0=-497.4746312593619, P1=9.446350825408219, P2=-32.83429800274731, P3=-11.179100408996101, P4=-16.373013616134628, P5=-12.120351402862363, P6=-11.637643601802734, P7=-10.80232878099552, P8=-3.4725366553876746, P9=-5.652445126839133, P10=-6.041348219566008, P11=7.93793023707303, P12=2.897976502090698, P13=0.8723264356537432, P14=2.4521683201368836, P15=-1.9574003808771512, P16=0.3105414012916875, P17=-0.8714353114829705, P18=0.531752795807507, P19=-0.8719788273710796, P20=-0.49447589825530863, P21=2.878640707587765, P22=-0.18778588399116275, P23=-2.8661309293829285, P24=-5.424105606397645, P25=-1.0226820151286715, P26=1.936981518420276, P27=2.0078178112162544, P28=-1.8130580709652762, P29=0.0868809815243985, P30=0.8759071041599901, P31=0.10276386410190623, P32=-1.5505519834149106, P33=-0.026696457510803684, P34=-0.8339563397462385, P35=-1.5942300286427864, P36=-1.9159863513304756, P37=1.8306787231301977, P38=-0.08453397668437869, P39=-0.7122209241252621, P40=0.6242149717136424, P41=0.6574578509560806, P42=0.6853061005983293, P43=0.6749000101048886, P44=0.7007991531726482, P45=0.6484346551877221, P46=0.6303469709060698, P47=0.6337228231437245, P48=0.6840397715383882, P49=0.7008485251405618, P50=0.6310076628243212, P51=0.6211140443332878, P52=7.443112251532541e-07, P53=2.762808827115507e-06, P54=7.951404007263688e-06, P55=2.1702382653035936e-05, P56=8.817354118925954e-05, P57=0.00016138012689545866, P58=7.421746756809192e-05, P59=4.2256450180613094e-05, P60=0.00018703462824682364, P61=0.0005878795010911427, P62=0.0017525078966039657, P63=0.0027782246742249234, P64=0.003045170585718046, P65=0.00644266499231392, P66=0.014174461976489194, P67=0.02071139192223026, P68=0.030329962233712923, P69=0.07985352905615099, P70=0.07378560899280222, P71=0.07109487738428834, P72=0.10213792362016946, P73=0.08708613682588633, P74=0.12602297903812315, P75=0.12538895775577627, P76=0.13271630335493692, P77=0.3083009582636115, P78=0.5074566462997534, P79=0.5175354756797271, P80=0.28695599495708224, P81=0.05115981030207052, P82=0.010236836480377959, P83=0.006152741157943133, P84=0.01320879409293661, P85=0.03214437961567583, P86=0.0642293349759359, P87=0.22847438469456616, P88=0.309857591513055, P89=0.1701780060415213, P90=0.22162732143392103, P91=0.332711802236311, P92=0.14088382313316905, P93=0.03392042807084954, P94=0.10037624102012714, P95=0.09366813542639577, P96=0.12819857778187294, P97=0.40757398528732885, P98=1.432386240855583, P99=2.3914118098996573, P100=2.4151970035394514, P101=0.9995649600443741, P102=0.06852168563992474, P103=0.1760071071543119, P104=0.32285171580921207, P105=0.07281919743800737, P106=0.030818672428034693, P107=0.029008249310109605, P108=0.05289882208950291, P109=0.20355292889865206, P110=0.3663577933092826, P111=0.5834859015109435, P112=0.2094968822576833, P113=0.2613997659328334, P114=0.06117532031436087, P115=0.073652814610456, P116=0.11917533212892244, P117=0.32091564228255665, P118=0.6472352922091893, P119=0.3225585245714577, P120=0.1501442215335933, P121=0.222360312010495, P122=0.11386588401323666, P123=0.0612778746491646, P124=0.02451439903160903, P125=0.03527347012298554, P126=0.02803888637374896, P127=0.009111929743019526, P128=0.014111554703522607, P129=0.018312461301854013, P130=0.009623263503759, P131=0.004489798756035773, P132=0.006345246656814592, P133=0.009249116217055084, P134=0.01863270841824778, P135=0.021741145902724347, P136=0.02275266564384605, P137=0.013223943221657375, P138=0.011890160924817128, P139=0.007299440484394814, P140=0.009505348908997562, P141=0.008394474865233599, P142=0.016737244343067545, P143=0.011455890586373668, P144=0.006466099625989909, P145=0.006631575013206275, P146=0.010504321987876807, P147=0.007142302772385868, P148=0.004964739010025492, P149=0.0034322001681301377, P150=0.003863831448028267, P151=0.0018512340476953184, P152=0.0018989211719945371, P153=0.0008820294258492855, P154=0.0002816044575894407, P155=0.00024871808758826947, P156=0.0002006688118910233, P157=0.000192044251892158, P158=0.00015947100279781827, P159=0.00020021124866151895, P160=0.00024888334584154907, P161=0.00024271997786908388, P162=0.00035746653493929904, P163=0.00030971281407853035, P164=0.000286670028495267, P165=0.00038560296928700767, P166=0.0004915990579978204, P167=0.00031708525011208425, P168=0.00023398656333650038, P169=0.00020652132262400554, P170=0.00025091875542747685, P171=0.00034102933610386947, P172=0.0003244234661255561, P173=0.00048403827573048364, P174=0.0006147410412943718, P175=0.0007333429723277662, P176=0.0009710473953099955, P177=0.0011776906650654382, P178=0.0008738168017950296, P179=0.00012977984307580183),Row( P0=-505.558507341193, P1=12.368617400554445, P2=-33.725429207997536, P3=-8.175722766615381, P4=-12.883970634091005, P5=-11.337237874176811, P6=-5.931350206110546, P7=-2.6248690770094965, P8=4.848074698309153, P9=7.947101312302642, P10=2.8112865507441773, P11=15.35679256241392, P12=7.844855767015037, P13=-5.054567937201955, P14=-3.8713134104102487, P15=-7.836165438753147, P16=-5.926316882521052, P17=2.78778055148156, P18=2.587659740655012, P19=0.8743891911826803, P20=-4.290590079683102, P21=1.6751461087709405, P22=5.712130950229066, P23=2.2073176036071187, P24=-2.453882956228147, P25=-5.598713460323393, P26=-1.9581833908177013, P27=3.531823428949776, P28=-3.96263220713441, P29=0.2934405574810666, P30=1.1243466194328526, P31=2.003839383747368, P32=-2.1129172804358216, P33=-0.07975731355216682, P34=-1.836151195417796, P35=0.9141086445230973, P36=-3.699080853355649, P37=0.027990655135886614, P38=0.06705979802061293, P39=1.0710271209157136, P40=0.5925368035275224, P41=0.5799647977829018, P42=0.5767252508944372, P43=0.5783231584186923, P44=0.563036665335136, P45=0.5666880970054946, P46=0.6191389600876983, P47=0.7073789409648186, P48=0.7875605951842567, P49=0.754948715277804, P50=0.6499302924919259, P51=0.5720296276277085, P52=0.0027429609259990158, P53=0.003911015010037837, P54=0.002608133963571094, P55=0.0007755223539497163, P56=0.0004546192319567667, P57=0.0003080508419870867, P58=0.00010817403265924427, P59=8.128422875927708e-05, P60=8.111580717294708e-05, P61=9.587533093115983e-05, P62=9.847175328238795e-05, P63=6.443873068716935e-05, P64=3.897571071165197e-05, P65=7.657417142643879e-05, P66=0.00010143326815928411, P67=0.00010545069547488599, P68=0.001026741744315108, P69=0.004392900396962768, P70=0.005102328369690505, P71=0.011205923845174033, P72=0.029134117610089345, P73=0.0445226611765784, P74=0.1769947341904366, P75=0.40990690170404603, P76=0.5789102342805198, P77=1.2607791638551527, P78=1.273002705890954, P79=0.6890944306916931, P80=0.25809253905435847, P81=0.06808768203703816, P82=0.045012138151129664, P83=0.04885921115282519, P84=0.06705067338238853, P85=0.16787305274101388, P86=0.5590180316327846, P87=1.622862085236812, P88=1.2649522556873871, P89=0.3016140252934731, P90=0.04528496198069375, P91=0.012507927962713418, P92=0.047217731280198336, P93=0.1277240809357294, P94=0.21048128627579063, P95=0.4456103691291379, P96=1.515804422947144, P97=1.9523580823258309, P98=1.6095860298077553, P99=2.1158516910505765, P100=1.7329410582544338, P101=0.502159769839795, P102=0.054077158583211826, P103=0.021496644631858138, P104=0.09604825507072429, P105=0.18129218318153476, P106=0.11135425860065534, P107=0.16936729095621633, P108=0.18735218733336037, P109=0.5644340835576525, P110=1.074390879352116, P111=0.49860910368217204, P112=0.2689850071187638, P113=0.06855890969174967, P114=0.04529822243695332, P115=0.041454262844145315, P116=0.09320028243015366, P117=0.5157687912211636, P118=0.6796967877353471, P119=1.269857676536353, P120=0.4805448156288575, P121=0.26056936614849086, P122=0.09983529393675153, P123=0.027190903559160286, P124=0.02880057662516314, P125=0.040649838109496655, P126=0.025201061365660887, P127=0.013243632701085096, P128=0.008816031917581561, P129=0.009429119282218913, P130=0.008011301659949064, P131=0.01040811492857179, P132=0.012277038477703877, P133=0.012729010722329267, P134=0.023402531151613764, P135=0.027052216179086663, P136=0.042430281001503624, P137=0.05925613122574669, P138=0.024400548054493498, P139=0.010041737769980939, P140=0.0074922898713555616, P141=0.010866348886879337, P142=0.01009540383336984, P143=0.005663553276697191, P144=0.004739330140151585, P145=0.008991119897299425, P146=0.009881612626172268, P147=0.005877633082769012, P148=0.0022327976283735168, P149=0.0006944925951641349, P150=0.0005653740022035705, P151=0.0005190366920102102, P152=0.0003860688304201, P153=0.00030440659141387497, P154=0.00047789050884152796, P155=0.0011075617160183446, P156=0.0010850578707512115, P157=0.0006512351606520218, P158=0.0007290056589884761, P159=0.0006325128618692085, P160=0.0006813242731157307, P161=0.0005954472168324316, P162=0.0004311535537142911, P163=0.0005699408131184785, P164=0.00041777504011708685, P165=0.0002839472306255073, P166=0.0002608409971710219, P167=0.0003745930302990434, P168=0.0005698307350101435, P169=0.00036176969347457823, P170=0.00016469995558330034, P171=0.00015717955187949885, P172=0.000150157817934745, P173=0.00023032334736007053, P174=0.0002477860695323427, P175=0.0003930954550517625, P176=0.0004960237212457642, P177=0.0004713391025415471, P178=0.00016543510281719954, P179=1.7273754995500233e-05),Row( P0=-517.5455167189335, P1=6.972058090085227, P2=-37.11193041237569, P3=-12.476074197060399, P4=-16.50572011519931, P5=-17.759778433201905, P6=-8.161368488145044, P7=-5.880135713215825, P8=2.9549370230905434, P9=8.370515578660864, P10=2.271593290512847, P11=17.152992368773468, P12=7.833347105729203, P13=-3.8397389595339093, P14=-3.557573404332177, P15=-9.766863296829499, P16=-4.548236815930236, P17=1.9616054547274406, P18=1.9913976487523524, P19=1.3273825645734156, P20=-2.355420334270497, P21=2.0725235890308586, P22=1.3818567773712673, P23=-0.08505422471616177, P24=-1.9095860937352385, P25=-4.255515380981927, P26=-3.4508070922914755, P27=3.8169350624426186, P28=-2.933764065677197, P29=-0.4104707691925197, P30=-0.8424880874895069, P31=1.331021189714136, P32=-0.46361971185316697, P33=0.6677134729324343, P34=-1.2113394233047758, P35=-0.6587141530357142, P36=-3.396583563821812, P37=1.0538991023234983, P38=-0.48300110151547304, P39=0.2991786258457713, P40=0.5809682665452846, P41=0.5435367127212064, P42=0.5242575229246865, P43=0.5500123258220864, P44=0.5435965619476185, P45=0.5766653405005391, P46=0.6695283010869865, P47=0.7017456681222255, P48=0.739948287041487, P49=0.6981062712021211, P50=0.623050259025684, P51=0.5543371479111361, P52=0.0026866469109969363, P53=0.005491434946418817, P54=0.004842362298169964, P55=0.0017279900774636351, P56=0.00042826582427966475, P57=8.03185783089693e-05, P58=4.878407040269492e-05, P59=0.00011312435662477605, P60=0.00016591209380849956, P61=0.00011748677642381815, P62=8.053494945871843e-05, P63=4.7580817213912076e-05, P64=3.1072763722839395e-05, P65=6.810947140906246e-05, P66=0.00026559483092484284, P67=0.0010195603231858577, P68=0.004874484529206446, P69=0.010493478217924733, P70=0.010806999495336096, P71=0.011410224784989266, P72=0.023846010610739734, P73=0.08729507122438994, P74=0.307335705903786, P75=0.34193456407806255, P76=0.27583148074088243, P77=0.5317130076962724, P78=0.6304379227273211, P79=0.4985092208011013, P80=0.30423822504337567, P81=0.11534781847451087, P82=0.35268970296809904, P83=1.2106917500326855, P84=0.6623870805906972, P85=0.11811067477542393, P86=0.010563632178512035, P87=0.005674269750911433, P88=0.0061177947310705255, P89=0.006418606721247745, P90=0.010256726206706222, P91=0.018310523738250206, P92=0.03814821821132685, P93=0.4637927161845965, P94=0.7928257480899372, P95=0.4467340798744748, P96=0.3263226076560873, P97=0.7312724782762103, P98=1.7700726697142635, P99=1.3695462154315852, P100=0.8638073709199642, P101=0.41831965834651325, P102=0.10412807288150785, P103=0.0027618054822200547, P104=0.00478180410538562, P105=0.04990060291681829, P106=0.08453525088417806, P107=0.04455813174398509, P108=0.04237411836815197, P109=0.13578815177911752, P110=0.20207033396409008, P111=0.1516722128299085, P112=0.07053024634358097, P113=0.1750671523887977, P114=0.20025118982724877, P115=0.15041625178737872, P116=0.09936729117387043, P117=0.14860947214909287, P118=0.1678378894381618, P119=0.7751595767064909, P120=0.3320151920409948, P121=0.048480279825543544, P122=0.10145143443899554, P123=0.05450931558773994, P124=0.019065038855083073, P125=0.023733191058798682, P126=0.034389325860694514, P127=0.05312799942415113, P128=0.0374924938585328, P129=0.0038130785048244554, P130=0.00249843619167924, P131=0.006100773815382004, P132=0.015041502425651224, P133=0.005572209586726215, P134=0.009841029798202512, P135=0.006930960138300256, P136=0.05553707335664589, P137=0.05427174690446654, P138=0.02085807541789462, P139=0.016949681372701978, P140=0.01874956040663574, P141=0.011630493461658068, P142=0.00981680718319894, P143=0.006254341519041681, P144=0.007679248167584887, P145=0.00828814669917111, P146=0.007975125385246671, P147=0.0031844088858871823, P148=0.0024123932005906314, P149=0.0016524873620822477, P150=0.0006134916035086809, P151=0.00044885687206861727, P152=0.00018668269423815275, P153=9.705008013560402e-05, P154=0.00014506782029174428, P155=0.000676578201234725, P156=0.0006387973237054907, P157=0.00027123489635098044, P158=0.00026208387209088034, P159=0.0002618924665134477, P160=0.00037064095361296045, P161=0.00043959227520807203, P162=0.00047038110385396386, P163=0.00046956629465196996, P164=0.0005955189612705931, P165=0.0005839247143722435, P166=0.000517499587669926, P167=0.0005000498445234414, P168=0.00041163887136271574, P169=0.000735441754150371, P170=0.00080511744539191, P171=0.0007557681963726134, P172=0.0007686926345339192, P173=0.0006768772427874906, P174=0.000492870607372979, P175=0.0006174317874421089, P176=0.0005792285853113461, P177=0.0007072483929885368, P178=0.0003973029129419852, P179=4.196970677768503e-05),Row( P0=-652.6727098808949, P1=38.45702820109734, P2=-22.79187427551314, P3=-5.147241658576338, P4=-14.92817479213601, P5=-15.703046077261503, P6=-10.784632277408527, P7=-10.912597459793046, P8=-3.385291562350311, P9=-13.249638515179464, P10=-9.786858429239226, P11=-6.0133344553964445, P12=-3.1569573749027224, P13=0.9745707900140341, P14=3.205774057199513, P15=6.146263037960241, P16=1.1711933180456344, P17=2.4023968715711606, P18=4.335943535541484, P19=1.2735556009238178, P20=-0.5636913512596564, P21=0.19342278770430024, P22=-0.5643752167543629, P23=-0.06932344056386187, P24=1.0271941296047944, P25=0.5227380796939074, P26=0.13830336725655148, P27=-2.0627688613244612, P28=-4.090149681999441, P29=-0.04854201986754601, P30=-1.190521068319654, P31=0.5929963388526198, P32=-1.69835906822251, P33=-0.6281411898573049, P34=-1.2044006833782046, P35=-2.0590924999676354, P36=-2.4621220118108145, P37=1.3964284473775572, P38=-0.24532137808198992, P39=-0.8001557832906834, P40=0.7009517822791119, P41=0.6764891205926566, P42=0.6637377094128603, P43=0.6638791265152957, P44=0.7323136087677472, P45=0.746470872831863, P46=0.6674921222207695, P47=0.6261078815012971, P48=0.6375056424020733, P49=0.6599751832981877, P50=0.6808160953235489, P51=0.6758115697104715, P52=1.1760926813559312e-06, P53=4.02759189617839e-06, P54=1.2033747034640862e-05, P55=1.981514043996236e-05, P56=3.7333198828220405e-05, P57=5.157102074384567e-05, P58=3.339349618683074e-05, P59=2.4964827389850687e-05, P60=0.00013207943246947048, P61=0.0012620898078441207, P62=0.012477516451508958, P63=0.01575803947862735, P64=0.012261474130645126, P65=0.07464569757055944, P66=0.12677680072947176, P67=0.1252773114972021, P68=0.07380009519481441, P69=0.05641730105915563, P70=0.034908934668230036, P71=0.06408550780256923, P72=0.15838081762938494, P73=0.11494118284161027, P74=0.025421909594439684, P75=0.009254403701332617, P76=0.004578188042611947, P77=0.004435373998967847, P78=0.006242730596577061, P79=0.03085330492521606, P80=0.08956432703338722, P81=0.05011440967466315, P82=0.01691701908909305, P83=0.017966882943306176, P84=0.02722967237775939, P85=0.035416653000200866, P86=0.03226445024077208, P87=0.021608520690782763, P88=0.007328750913405148, P89=0.005529875311955634, P90=0.00881608696621333, P91=0.023519254461569224, P92=0.03598197667548045, P93=0.03619590516850805, P94=0.016676709717738908, P95=0.010601534353277509, P96=0.0038256170047534593, P97=0.008801695678943075, P98=0.010289493197120803, P99=0.004656809206657753, P100=0.004117763847498498, P101=0.0043262685457949, P102=0.0017728098961067658, P103=0.0011750314071252597, P104=0.0011061689814357623, P105=0.0039418938760408286, P106=0.013168764403095327, P107=0.009270212506628593, P108=0.007436294825505025, P109=0.01310704427581019, P110=0.01466834704987129, P111=0.009713211770619068, P112=0.008757296955855124, P113=0.02434931538970468, P114=0.016831086458724404, P115=0.008448637120563027, P116=0.0028968125640297616, P117=0.002251635688426795, P118=0.018530686796936403, P119=0.03931303238345489, P120=0.017656734818655394, P121=0.0077531308819156364, P122=0.00424110508870504, P123=0.013659595176234815, P124=0.007669773575911606, P125=0.0021050832042707793, P126=0.0011761305901096084, P127=0.0010256158978167685, P128=0.0003081244214744554, P129=0.0002697838886904063, P130=0.0004383751091840759, P131=0.00022567787745061353, P132=0.00011773383440471021, P133=0.00011610433436393817, P134=0.00032604692436943877, P135=0.0003003252790637616, P136=0.0001377098197370231, P137=0.00022761834111529225, P138=0.0005371911940649354, P139=0.0004508103758771515, P140=0.00037603218446455414, P141=0.0003812128443193558, P142=0.0002294045046900009, P143=0.00018166233770206366, P144=0.000185009251636131, P145=0.0001045051841215668, P146=6.319794734314578e-05, P147=6.237144034818718e-05, P148=7.260845756537203e-05, P149=8.809829666212988e-05, P150=9.182768374977788e-05, P151=0.00011591063037992051, P152=5.9994393011966844e-05, P153=3.9506772282500724e-05, P154=2.9561335598627878e-05, P155=2.7882072285958593e-05, P156=4.586489237873798e-05, P157=6.180243658865496e-05, P158=0.00011602357015283694, P159=0.0001089381297717115, P160=9.724133552462752e-05, P161=7.36537366883571e-05, P162=3.627797749089555e-05, P163=3.489621134723321e-05, P164=3.845262789908378e-05, P165=4.51288591571317e-05, P166=3.241677233279404e-05, P167=1.933861276686459e-05, P168=1.9582312779527308e-05, P169=1.894796858615834e-05, P170=1.8219488148386286e-05, P171=1.603586412520853e-05, P172=1.619175259844305e-05, P173=1.1117919501906215e-05, P174=1.0384542624086893e-05, P175=1.2348257717322256e-05, P176=1.706429687983788e-05, P177=2.949074691047417e-05, P178=2.0138467468773515e-05, P179=2.432379478162054e-06),Row( P0=-540.712170614211, P1=6.969968539305422, P2=-36.02662929635067, P3=-6.209252571166623, P4=-21.582723205206747, P5=-10.94995054039161, P6=-14.384984014881365, P7=-11.224886851789789, P8=-0.3431096902270422, P9=-2.7378464774813738, P10=-3.555838371966892, P11=12.257979250848237, P12=5.641562097348645, P13=1.0630004024705613, P14=-1.8846956913142665, P15=-6.015888079649609, P16=-3.7354718865134435, P17=-0.9262929721433854, P18=4.813046650401626, P19=2.0826007943035623, P20=-1.9216599480590837, P21=1.0976279062004426, P22=2.0134323908783234, P23=1.6382126114471307, P24=-3.0957405884954965, P25=-3.757224547042645, P26=-1.7705216869772167, P27=1.3362965960799615, P28=-0.9457854765358861, P29=1.0964034421888265, P30=0.3194511971632249, P31=0.5722895601884072, P32=-0.019228928080539096, P33=0.3336543124125263, P34=-2.4248746237514576, P35=-0.7415831324154049, P36=-1.6077317913480007, P37=1.5991898354497087, P38=-0.307717432502655, P39=0.0784269273601466, P40=0.5708526844336536, P41=0.5825703100165259, P42=0.5921791186864993, P43=0.549320073955303, P44=0.5678151396436981, P45=0.6006853169539661, P46=0.668272384865727, P47=0.7183379409523415, P48=0.7715120904385203, P49=0.7108041651074901, P50=0.5984602051339483, P51=0.552052497469554, P52=5.230458839486767e-06, P53=6.799824939869242e-06, P54=7.1360012540227365e-06, P55=1.6076662033753772e-05, P56=2.8720971370321912e-05, P57=3.7791254373863605e-05, P58=8.538607310260638e-05, P59=0.00015643395699018085, P60=0.00022718356503682164, P61=0.00027536610248363755, P62=0.00047780508628505027, P63=0.001494534955003557, P64=0.002399838909949543, P65=0.003874720910995613, P66=0.003511953850213996, P67=0.003466807245086417, P68=0.006941299595804576, P69=0.020440808878703257, P70=0.020339818264191987, P71=0.018440474536442986, P72=0.03904183997259796, P73=0.05700727397834482, P74=0.19846718726252946, P75=0.19326323385750596, P76=0.19527467871092538, P77=0.4327151480536799, P78=0.5476445096126378, P79=0.6056580413156136, P80=0.2306146496154185, P81=0.047935482563208544, P82=0.00858626494638003, P83=0.0012914261606956057, P84=0.0015220108570903955, P85=0.0023880007013290217, P86=0.002585658631235239, P87=0.003251046573293594, P88=0.004539065986602152, P89=0.006429323048463124, P90=0.01668332218782602, P91=0.06527510365005887, P92=0.0970482307331047, P93=0.11932603944779681, P94=0.4742440257196609, P95=0.5942240003607931, P96=0.1367598401632091, P97=0.17606649350339285, P98=0.3147771743361897, P99=0.21298537744748838, P100=0.2387305260548257, P101=0.1291899509764429, P102=0.12327444225052242, P103=0.053030530445972064, P104=0.03872294760519431, P105=0.02243055117854554, P106=0.10709485932035834, P107=0.4039081010376725, P108=0.2615467714172467, P109=0.5795637348171347, P110=0.7328903990272154, P111=0.42040618779493144, P112=0.10861825312969194, P113=0.0425065328866101, P114=0.028451790853134436, P115=0.08310298858443545, P116=0.2959018827738402, P117=0.26263893305533664, P118=0.11739834981740249, P119=0.2753011025259324, P120=0.10259770708980384, P121=0.04646089874368812, P122=0.036846828607031826, P123=0.13723712450858805, P124=0.05098216843914771, P125=0.006189693783323796, P126=0.007759315884764559, P127=0.02307424034834972, P128=0.023167624540580824, P129=0.0046337539335944925, P130=0.0023524928847075484, P131=0.004895594332523277, P132=0.004483263282677222, P133=0.007107487919854188, P134=0.009107110963832044, P135=0.0031760711961178437, P136=0.014431544684440283, P137=0.07860433587087413, P138=0.04207052918908999, P139=0.021912754566027783, P140=0.026905104331328335, P141=0.011731903422476214, P142=0.005411291553996972, P143=0.0066753471222662205, P144=0.029187855323436946, P145=0.012134821075261867, P146=0.017070841079393856, P147=0.015855355456263473, P148=0.012928226802774117, P149=0.012018999877958436, P150=0.0034518582497328793, P151=0.0030269234770152103, P152=0.004302827771954528, P153=0.000629936445359965, P154=0.0004633379912270449, P155=0.0005506448283494244, P156=0.000665480570623098, P157=0.0005373790089342781, P158=0.0006545104372576068, P159=0.0005799137765337397, P160=0.0006605454379803334, P161=0.000524668875867504, P162=0.0006751904198289549, P163=0.000855380267825498, P164=0.0007004330901525123, P165=0.0007381944665250905, P166=0.00042796181818368327, P167=0.00033521881557688703, P168=0.00039172294988123145, P169=0.00034444577712571694, P170=0.0004927087379057093, P171=0.000539706016148207, P172=0.00036909856454907486, P173=0.00026523783807831, P174=0.000182019828450707, P175=0.00015887132972539322, P176=0.0002322800591620484, P177=0.00021617372270559728, P178=0.00016502617352514917, P179=1.4254782651623197e-05),Row( P0=-720.8184844699362, P1=50.81535239274539, P2=-10.254295719708338, P3=0.2614421879804982, P4=-14.254512550418436, P5=-9.995554358507174, P6=-6.042418191979517, P7=-9.496367959359873, P8=-0.13360464794672652, P9=-13.068567664634914, P10=-9.888557352356925, P11=-8.729045103028298, P12=-8.581570095098941, P13=-10.184161613391444, P14=-5.20445529633283, P15=-0.5659452441240047, P16=-5.446922348922765, P17=2.6304037758097563, P18=4.306123289785649, P19=4.055876786226382, P20=3.5525891681970037, P21=6.035178237109824, P22=5.906849717655425, P23=4.390745048003938, P24=5.799085811265536, P25=2.7448361248492024, P26=0.41462639220671593, P27=-1.6060771157497051, P28=-5.027270764790943, P29=-1.1672939647058032, P30=-0.7922588036785846, P31=0.7497412182203786, P32=-1.5815243267414583, P33=-2.3041609297555743, P34=-4.720876953765859, P35=-0.8444931859506288, P36=-2.944825698366607, P37=2.4447990914285413, P38=-2.5672157076926116, P39=-2.4545348950164083, P40=0.6362928641359654, P41=0.6331931406333409, P42=0.5985969571327413, P43=0.5910376766882158, P44=0.6693436489870096, P45=0.7168354538416251, P46=0.7447034478184125, P47=0.6872536986978146, P48=0.6573811182296095, P49=0.6707850523875935, P50=0.6927070846212595, P51=0.6486941251553557, P52=5.809686620287486e-07, P53=2.3492575287144437e-06, P54=6.760626128133816e-06, P55=1.7932108132112697e-05, P56=3.081097207608279e-05, P57=3.612227278495601e-05, P58=1.7569544844873195e-05, P59=2.386574003947749e-05, P60=0.0006138337035471021, P61=0.008701704428202848, P62=0.046716020237259126, P63=0.05047822918721967, P64=0.021178299483858427, P65=0.051383216970360725, P66=0.04928861727316543, P67=0.01279073169048826, P68=0.0013741923140836514, P69=0.00024690961790585925, P70=0.00011335822541267379, P71=0.00030775840991397244, P72=0.0011363587475873259, P73=0.002009598483248079, P74=0.008056937392494374, P75=0.007989634205883804, P76=0.003104987848949629, P77=0.0021513685127369352, P78=0.005523610505972188, P79=0.009537059307917636, P80=0.008140635777777136, P81=0.003967125991982204, P82=0.0015647110593386853, P83=0.001326685815992083, P84=0.002350590314126099, P85=0.005244021464695355, P86=0.004438776665089543, P87=0.002508432589018496, P88=0.0010148848643086823, P89=0.0003861507599071279, P90=0.0014895803969549792, P91=0.005963828157780279, P92=0.0027605832648179723, P93=0.0020234766733032428, P94=0.0034032236060878886, P95=0.0016556370890708132, P96=0.0001486466329558201, P97=0.00011209627498690193, P98=0.00027297504833052644, P99=0.00026422712972818557, P100=0.00031632604146648895, P101=0.00034310760366727263, P102=0.0006825602339566358, P103=0.0003107011120489032, P104=0.00011593465960488677, P105=0.00016606094861050264, P106=0.00011988147324086044, P107=8.655005855483354e-05, P108=0.00012115775076679765, P109=0.0006018876024814202, P110=0.0002366639096246787, P111=6.587649188294967e-05, P112=7.197563454994848e-05, P113=3.049607765685921e-05, P114=0.0002066755721759631, P115=0.00024170416482194993, P116=7.159640542510753e-05, P117=5.146589313625726e-05, P118=4.8192011888994416e-05, P119=0.00022353304184785026, P120=0.00023143633269214208, P121=8.738576015430481e-05, P122=7.621836085026805e-05, P123=9.129089892989413e-05, P124=3.98232648329868e-05, P125=3.9997832269640165e-05, P126=2.0181276202807928e-05, P127=1.3870994195413118e-05, P128=1.2477207947343555e-05, P129=9.101002800314288e-06, P130=8.035619272527565e-06, P131=3.7610612979318285e-06, P132=5.2186743706711335e-06, P133=5.333025734349451e-06, P134=6.437223138958525e-06, P135=9.402337505005203e-06, P136=1.045305646804834e-05, P137=1.0526001969405174e-05, P138=1.8138187178316722e-05, P139=3.0071452898158962e-05, P140=3.913666049386167e-05, P141=2.4469494672291165e-05, P142=1.739750808488323e-05, P143=1.2674541187609198e-05, P144=9.97586589829853e-06, P145=1.0160600361356145e-05, P146=1.1991909176373694e-05, P147=1.5838373460320835e-05, P148=5.2486482565624876e-05, P149=2.276066608536691e-05, P150=7.807781844099089e-06, P151=9.864158358121244e-06, P152=9.664673366013504e-06, P153=5.040925325309053e-06, P154=5.728185417234598e-06, P155=7.1733682746045745e-06, P156=6.852858541705425e-06, P157=6.122713656260872e-06, P158=5.460006445322064e-06, P159=4.690382543515208e-06, P160=5.082118601873897e-06, P161=3.6982153151890037e-06, P162=2.8678122970895567e-06, P163=2.623690563644634e-06, P164=3.4194739255612946e-06, P165=2.9946829106007157e-06, P166=1.7731384655963962e-06, P167=2.8321929484700643e-06, P168=2.4790883806929804e-06, P169=3.215139279625181e-06, P170=4.427917173478014e-06, P171=4.3110821362743155e-06, P172=3.2641117264606027e-06, P173=3.477075400054791e-06, P174=3.968044574798049e-06, P175=4.647187380979929e-06, P176=8.080422640190319e-06, P177=6.283170676140876e-06, P178=2.6560069999258474e-06, P179=3.97521482423365e-07),Row( P0=-599.2761331683238, P1=42.285529857634955, P2=-25.066638178278556, P3=-9.30423032983477, P4=-9.661296484276459, P5=-13.639173918843516, P6=-7.003700010120236, P7=-9.531283823746781, P8=2.650372583180249, P9=-11.157773198370748, P10=-12.059674773942506, P11=0.44296131448440934, P12=-0.14675073200361644, P13=4.517156893131336, P14=13.778981746406917, P15=9.415831798840227, P16=9.120037324969868, P17=2.6231662280106214, P18=3.2855283874121053, P19=2.664067476668406, P20=-5.610185919444557, P21=-0.34093393637524483, P22=-0.30143033820476356, P23=0.3648016936343622, P24=-1.0079417103893245, P25=-6.6970436347215765, P26=-5.706162453602786, P27=-1.3405747899991938, P28=-1.8105915594065156, P29=1.5401788203893387, P30=-1.904666567674157, P31=-0.34316833106537725, P32=-2.338455228890056, P33=0.1838699936336266, P34=-1.3462405769586807, P35=-1.2253293653012884, P36=-4.3256873208036755, P37=2.078442797772136, P38=0.047126246823250215, P39=0.24222451047741003, P40=0.7418243031721643, P41=0.7021607862303282, P42=0.6354925863746558, P43=0.5781500158350535, P44=0.5893274518657676, P45=0.5804915285532112, P46=0.588145694650532, P47=0.621827168113474, P48=0.6295259898886014, P49=0.5813956350770818, P50=0.5927969953684791, P51=0.6487031348300482, P52=0.0029475047110276644, P53=0.0023645114425266196, P54=0.0010655671980623533, P55=0.0002760872208885617, P56=0.0001621654722243356, P57=9.528749648839416e-05, P58=4.530886097595403e-05, P59=2.8216891666952143e-05, P60=3.635594901809819e-05, P61=4.860645925637231e-05, P62=0.00032694581963156936, P63=0.001612880285192856, P64=0.005634473643507773, P65=0.026278737054919746, P66=0.062052947507674004, P67=0.06910307357540821, P68=0.17993486736082748, P69=0.2284355890862232, P70=0.09967310378568403, P71=0.11620229837479744, P72=0.17160719762457144, P73=0.09350019495192416, P74=0.09883432684599289, P75=0.14798099425237682, P76=0.1496221678775356, P77=0.058234148544711296, P78=0.008823680742332739, P79=0.0021277700551655455, P80=0.003390562734732901, P81=0.008256992195857687, P82=0.008797861664194032, P83=0.012188731610235645, P84=0.05223777675903609, P85=0.16298768018670887, P86=0.19694713311425358, P87=0.15647548604059497, P88=0.04657102183097654, P89=0.013023296200444045, P90=0.018908132962858083, P91=0.04179260540715889, P92=0.026354985587826098, P93=0.03457424250262077, P94=0.04857043535524069, P95=0.031232684887819377, P96=0.010964065910955846, P97=0.016372277956090937, P98=0.024607617214299247, P99=0.011802411012641704, P100=0.005257627066988146, P101=0.0006335170317482789, P102=0.0016239767139734464, P103=0.002554157466277658, P104=0.0030708915607284134, P105=0.006587249912148916, P106=0.012376471194201688, P107=0.009853861480254753, P108=0.006009086441855355, P109=0.007620268816361755, P110=0.0033046554006684764, P111=0.0018978883198253048, P112=0.002129880634971061, P113=0.003545359055592036, P114=0.00561875423176107, P115=0.0049861831089038355, P116=0.004025403133359738, P117=0.005340840333318164, P118=0.0027106877422073085, P119=0.007151526237437131, P120=0.00750213847230472, P121=0.001219627078230591, P122=0.0009980508738278266, P123=0.002218286462664624, P124=0.004278021765847614, P125=0.0029349330315421476, P126=0.00033731006814510334, P127=0.00034765734399999935, P128=0.00040304767698931033, P129=0.0010187861430493456, P130=0.0010928497083433316, P131=0.000331650170694517, P132=0.00019664572501020583, P133=0.0002066570037390711, P134=0.00036709710837028696, P135=0.00031767221657952907, P136=0.0002187822050802704, P137=0.00028601040349004347, P138=0.0008936793343963295, P139=0.0014825245366319928, P140=0.0007021539368759353, P141=0.0003791308747246312, P142=0.0003222172554888149, P143=0.00028383840183069514, P144=0.0002371902173353287, P145=0.00039889895613557947, P146=0.0002379272311619681, P147=0.00012304137651259525, P148=0.0001973269087057823, P149=0.00023334431093972383, P150=8.04736827129404e-05, P151=4.809298366181373e-05, P152=4.1280542816863864e-05, P153=1.8604924031933143e-05, P154=1.786724902648501e-05, P155=2.851855419263402e-05, P156=5.923390308944707e-05, P157=5.770501628770419e-05, P158=7.633942337424324e-05, P159=8.229870620382264e-05, P160=0.00015006319789040627, P161=0.00022232205076698334, P162=0.00015798073492928353, P163=0.00015010468083963952, P164=0.0002591380403489031, P165=0.00011866789459270118, P166=0.00014170363737811927, P167=0.00011663607906387051, P168=0.00010689606698659208, P169=0.0001752655477295804, P170=0.0001611224789695548, P171=0.00023294790824405386, P172=0.0002964267066779542, P173=0.00041014623629067384, P174=0.0003882968101809507, P175=0.0005391679469203819, P176=0.0003461477234286342, P177=0.0005702247774157817, P178=0.00029509084435768915, P179=2.545267050428917e-05),Row( P0=-457.6475646153594, P1=-1.7304933179512394, P2=-32.163404516195065, P3=-8.350568392530167, P4=-13.137202261012888, P5=-10.040656625809364, P6=-2.242359826401727, P7=4.963652596401883, P8=13.733148475002348, P9=15.545059027219699, P10=2.66677060563163, P11=1.9334007682815595, P12=-10.860199484908987, P13=-5.494766880388305, P14=3.361776864566591, P15=8.108262505547867, P16=2.175149492276124, P17=-0.4471695586004018, P18=-1.0938069433427335, P19=1.6389000848638062, P20=-1.0443544378918619, P21=-0.1109066870424477, P22=1.7818704146423459, P23=5.176142711554747, P24=0.19041353529667276, P25=-3.9427893801499994, P26=-0.9797817419330034, P27=3.853753057562954, P28=-1.2863038513296208, P29=1.4145716968050956, P30=-0.7409144633606807, P31=1.521018777829456, P32=-1.5969477906497285, P33=1.1744150338676538, P34=-0.3137955728185536, P35=-0.7038279979934696, P36=-0.8802041993947065, P37=2.9429623206193374, P38=-1.1820239723343025, P39=-0.06811985253777289, P40=0.7221264607992149, P41=0.72362672087596, P42=0.6523693471171408, P43=0.5522569376937808, P44=0.49053641405827764, P45=0.49797216144592144, P46=0.517077858817831, P47=0.5447254470895112, P48=0.6295357391968838, P49=0.6616080775742657, P50=0.6502189493990183, P51=0.6754049940161854, P52=0.004532643991108116, P53=0.004940361628423318, P54=0.00223710382089646, P55=0.0007447421178883842, P56=0.0002146153296828553, P57=0.00013355382101129542, P58=6.529157011854953e-05, P59=6.037118017854579e-05, P60=6.270919044002353e-05, P61=6.740361823954039e-05, P62=8.83891029813176e-05, P63=6.752594750723221e-05, P64=5.1678162174931405e-05, P65=0.00010020108515308197, P66=0.00011970788725107834, P67=8.613817108471511e-05, P68=9.644068442768782e-05, P69=0.0002295891655515324, P70=0.0004975536195446179, P71=0.0035670954540579173, P72=0.013044591423731575, P73=0.020664424912994594, P74=0.049958717956566534, P75=0.0806617200542045, P76=0.11688773202710692, P77=0.3249160039095633, P78=0.5282304099620118, P79=0.7028591156243246, P80=1.2271814576768638, P81=0.9610291772161679, P82=1.1057096838920577, P83=1.57880679875091, P84=1.2949895768288568, P85=2.0123460570753284, P86=1.075735999273567, P87=0.7844534019301322, P88=1.1280072838392041, P89=0.30674022894025826, P90=0.008947845647502681, P91=0.002684722529872032, P92=0.0011790957394769105, P93=0.0004106619342766635, P94=0.0006363175854371448, P95=0.0006576072227020717, P96=0.000826150002141976, P97=0.020892269660964868, P98=0.10195206909147517, P99=0.08749963340973486, P100=0.09028078300849643, P101=0.11535342818736957, P102=0.08179699645557763, P103=0.08834474466843248, P104=0.14109657543315812, P105=0.23275764282818837, P106=0.31862615883009554, P107=0.5108279233183828, P108=3.524543400967773, P109=26.465603324757915, P110=7.206618042230349, P111=0.18014408409987753, P112=0.1238955690266614, P113=0.11962530639047268, P114=0.18907780949917397, P115=0.19779219278905272, P116=0.14286871356562947, P117=0.15862573410283223, P118=0.251184318552654, P119=0.9291014623094763, P120=0.8004709767755442, P121=0.24369092157096325, P122=0.04906006893203077, P123=0.0298032732437167, P124=0.02219676460116748, P125=0.044814260716711476, P126=0.11049587433878857, P127=0.06090030419144888, P128=0.1476045403949247, P129=0.08841378208630063, P130=0.031058796104879507, P131=0.011557914417086087, P132=0.013406244097854573, P133=0.008280908719508663, P134=0.019593492061672275, P135=0.026381857208543264, P136=0.013929968659709137, P137=0.018454892457994573, P138=0.028293637252519543, P139=0.016109021640492212, P140=0.03995534392902266, P141=0.04282067557694725, P142=0.033517746095003076, P143=0.014164078045728551, P144=0.010793495113492793, P145=0.01078228706881995, P146=0.006637237234827808, P147=0.007600609417652192, P148=0.008092532919452455, P149=0.007713185249012951, P150=0.005149694837269725, P151=0.0017050269938686338, P152=0.0014542827879165882, P153=0.0013183479344703111, P154=0.0010742759067701236, P155=0.0011264288842174652, P156=0.0013247815232524634, P157=0.001958526253251376, P158=0.002041389581294199, P159=0.002102188162189787, P160=0.002346796714945518, P161=0.001390861649349862, P162=0.0017627123368816376, P163=0.0024692367326494807, P164=0.0014208325579153393, P165=0.0006990374923011876, P166=0.0016247135847025013, P167=0.0019575448749378755, P168=0.0012105334297796318, P169=0.0012190925619075899, P170=0.0007221865173879442, P171=0.0007607152263317447, P172=0.0006751711552196253, P173=0.0005951519426202049, P174=0.0007470255710813569, P175=0.0007286301489101424, P176=0.00041266112616918766, P177=0.0003600335346109442, P178=0.00019367796391049966, P179=1.9014765206871077e-05),Row( P0=-697.4876830524748, P1=48.81827767058841, P2=-2.3081985209008034, P3=-7.485928414801098, P4=-9.781659418997666, P5=-13.042792615159373, P6=-8.992776210001859, P7=-8.941586195684309, P8=-1.293892827371412, P9=-11.796180724391093, P10=-10.347308226694189, P11=-9.369253024632723, P12=-10.090835638860625, P13=-7.836244722977328, P14=-5.61275033070368, P15=-0.23951223418325054, P16=-2.7374121274983683, P17=2.8990790121578214, P18=4.577843779413068, P19=4.621152380767505, P20=4.9785657074032645, P21=3.668690898359521, P22=2.8818224152830303, P23=3.9744679799281877, P24=3.444923290695001, P25=3.765705169288135, P26=0.23979666333026858, P27=-0.8501252503248796, P28=-3.963804285875768, P29=-1.0612258607044762, P30=-1.2808073854928161, P31=0.4712189344559336, P32=-1.4388683347541138, P33=-1.5979848046901075, P34=-3.5266973711920064, P35=-0.06300149842363494, P36=-2.0185136082700437, P37=1.6820889990885268, P38=-2.482179600442633, P39=-2.7984722355151486, P40=0.6516598256930064, P41=0.6498415013104931, P42=0.6058698996730468, P43=0.6198476626713321, P44=0.6640514223850276, P45=0.6941886952544811, P46=0.6926618433380968, P47=0.6408187070492337, P48=0.6151710997503926, P49=0.63130545634055, P50=0.6798233431160623, P51=0.6624696451987114, P52=5.902883614995304e-07, P53=1.3436843669170315e-06, P54=6.743991305324215e-06, P55=1.6870306977334604e-05, P56=2.7147733119352717e-05, P57=3.344987601792956e-05, P58=2.076010942207919e-05, P59=7.737351581508007e-05, P60=0.0018958773561434125, P61=0.011448010589132567, P62=0.045793971712602716, P63=0.05591954708583046, P64=0.027098516105417875, P65=0.07998152241314876, P66=0.10730298777498769, P67=0.04336233653494508, P68=0.01766432727692401, P69=0.0033665615984978447, P70=0.001022193411806761, P71=0.0009002479088552501, P72=0.0014417605622917245, P73=0.0028233920773899643, P74=0.010794219078565006, P75=0.008406006223273293, P76=0.002535619791672812, P77=0.004723418775595571, P78=0.009751265731754982, P79=0.011481738731935417, P80=0.018628961700708095, P81=0.013249672115060568, P82=0.006682931822402795, P83=0.013793453783902308, P84=0.034158822866773055, P85=0.025393539284449165, P86=0.008147818038626293, P87=0.0021990519484799665, P88=0.0007498091259712799, P89=0.0009525235354633317, P90=0.0022472892031454223, P91=0.003207646614889965, P92=0.004011779880999795, P93=0.005689715400231506, P94=0.002367338666091402, P95=0.001573688342427536, P96=0.00023099500514453002, P97=7.383999308677442e-05, P98=8.789026629425318e-05, P99=0.00011399801168816892, P100=0.00021678970028404554, P101=0.00276510140219842, P102=0.0028976831460878543, P103=0.0002975847843570566, P104=3.623639142144644e-05, P105=0.00012289252311625183, P106=0.00012659328450765483, P107=0.0001320860214161541, P108=0.0011596360500784684, P109=0.0007493662751012031, P110=6.570944697867458e-05, P111=0.00013397719148773194, P112=8.512879235502048e-05, P113=0.00020005149334216925, P114=0.0006491068407132646, P115=0.00024006130205693284, P116=3.9729558509339964e-05, P117=3.378673291712443e-05, P118=0.0008409527056127853, P119=0.0012726566199710082, P120=0.00022812606155135336, P121=3.3416386785563166e-05, P122=0.0003499787253428998, P123=0.0004407929080704806, P124=6.660031739786142e-05, P125=3.286822003739605e-05, P126=5.148498460271212e-05, P127=3.0481879959217256e-05, P128=1.3013288463250252e-05, P129=1.1489879745870426e-05, P130=1.9812211278314233e-05, P131=1.1101159570682062e-05, P132=5.5312293735501295e-06, P133=1.1892237323045592e-05, P134=7.63502229459399e-06, P135=8.815520185456673e-06, P136=7.595343737487428e-06, P137=1.2098698882898355e-05, P138=1.1127433693328353e-05, P139=1.8533745245117174e-05, P140=2.425216775415307e-05, P141=1.7155720832492096e-05, P142=1.249185430925401e-05, P143=1.3488734604823988e-05, P144=5.018822614302161e-06, P145=5.58215917889481e-06, P146=6.522644027698825e-06, P147=7.828811645816829e-06, P148=8.838126745894153e-06, P149=1.3756089926313525e-05, P150=9.124858613232353e-06, P151=9.373433778973751e-06, P152=7.178810628317512e-06, P153=7.129659689924364e-06, P154=8.515937965512183e-06, P155=1.0099999521301092e-05, P156=1.7420641639635805e-05, P157=1.6483438887672398e-05, P158=1.6994846820737444e-05, P159=3.8838388180659223e-05, P160=4.8228011587266736e-05, P161=3.123693137118604e-05, P162=2.4999852105758775e-05, P163=3.899668715416191e-05, P164=4.119309287051433e-05, P165=3.717008416079162e-05, P166=3.114601310645304e-05, P167=5.80977148599226e-05, P168=8.233655235716065e-05, P169=4.904223731123135e-05, P170=3.785865251486378e-05, P171=3.666745892715121e-05, P172=2.9183629796339935e-05, P173=3.530687910907082e-05, P174=6.928550648309438e-05, P175=9.194820498310289e-05, P176=5.5624591832206304e-05, P177=6.989181015022297e-05, P178=4.334481833288452e-05, P179=4.983602994353898e-06),Row( P0=-729.2476906574861, P1=50.92939870773715, P2=-3.2699897641124074, P3=-1.7157522495898347, P4=-14.939336205188187, P5=-10.894636893667181, P6=-6.200242200988299, P7=-9.460395163078996, P8=-1.3303815733144142, P9=-10.5129014449329, P10=-10.477194916935936, P11=-8.061271174973172, P12=-9.783547177884326, P13=-9.231360762916403, P14=-6.770180037167882, P15=-2.162939325064232, P16=-5.2486190558513375, P17=1.0819342375047416, P18=3.540804035047023, P19=3.3746956056104875, P20=4.373461066401098, P21=5.006451371006768, P22=5.698317040035283, P23=5.340075714997046, P24=3.0645331061017935, P25=3.3438532620939996, P26=1.6718694508266598, P27=1.2664016085973742, P28=-3.161384056744452, P29=-1.5170687215890024, P30=-2.760830024977901, P31=0.6011229294080055, P32=-1.5061569622431275, P33=-0.24090251022356332, P34=-5.864116713314761, P35=-1.16105654038965, P36=-3.9976033950705627, P37=2.5484188534022874, P38=-2.1677686931676545, P39=-2.23927653957231, P40=0.5820495111700763, P41=0.6022213751847233, P42=0.5813607674853163, P43=0.5984585703934666, P44=0.6650063117999762, P45=0.7150964708582204, P46=0.6849689059686822, P47=0.6395031720161236, P48=0.596717306017203, P49=0.6198834337422731, P50=0.6576808545659419, P51=0.6142347756485037, P52=3.9503985241901006e-07, P53=9.68866769585489e-07, P54=4.993882797429102e-06, P55=1.5482591656884817e-05, P56=2.3844801396167937e-05, P57=2.057117638057562e-05, P58=1.0482090509325502e-05, P59=6.042291939992049e-05, P60=0.0020350997348297325, P61=0.011752529343999908, P62=0.035463649314525524, P63=0.037850501592125725, P64=0.022075449960302126, P65=0.05478624152768845, P66=0.030053905371915778, P67=0.0033485300440754076, P68=0.0005573039891646727, P69=0.00016351037562145954, P70=0.0001400124339034523, P71=0.0002574612113257306, P72=0.0006060332177847512, P73=0.0013545379734357973, P74=0.003071189223168441, P75=0.004090971201980653, P76=0.0024538236774247412, P77=0.002530861322773348, P78=0.005645188338657872, P79=0.008500236474339439, P80=0.009268167904844478, P81=0.002440008814182458, P82=0.0005947175680387934, P83=0.0006670243418495635, P84=0.001832096839754433, P85=0.003048316541276663, P86=0.0012646305767921413, P87=0.001426685243420987, P88=0.0005641744045395466, P89=0.0005760358607749818, P90=0.0017875704728970792, P91=0.0022476324281960574, P92=0.0011894049709450063, P93=0.0005457871417089855, P94=0.0005469841009149719, P95=0.0001847200327296794, P96=5.666075769462034e-05, P97=6.126367938151883e-05, P98=6.790087691095899e-05, P99=0.00018013271395126858, P100=0.0008742914849854522, P101=0.000678386771693989, P102=6.171079520687718e-05, P103=2.2925269239030584e-05, P104=1.2378110030949262e-05, P105=2.5460578951110008e-05, P106=6.779515882030335e-05, P107=0.00045460958673343617, P108=0.00026012059250577575, P109=3.578936171707909e-05, P110=3.7495695467212966e-05, P111=4.5246533406723684e-05, P112=6.748004709912806e-05, P113=7.817914901315881e-05, P114=2.3247234934756527e-05, P115=2.1409035851888943e-05, P116=1.5004062306660681e-05, P117=5.766026445569963e-05, P118=0.00015568029771705202, P119=5.3308800422664054e-05, P120=3.0699690662398047e-05, P121=3.773165595509307e-05, P122=4.9430387545982786e-05, P123=3.158535050664886e-05, P124=2.007064392267473e-05, P125=1.4333248654540936e-05, P126=9.82065300037645e-06, P127=1.0295847256827614e-05, P128=7.225368919862715e-06, P129=4.009263651795635e-06, P130=4.394535583275431e-06, P131=2.392198879443697e-06, P132=2.9564750905285096e-06, P133=2.0880058938018296e-06, P134=2.968307179303418e-06, P135=3.72287578668902e-06, P136=2.8916524802080916e-06, P137=5.617926420282246e-06, P138=7.863797466639683e-06, P139=1.0737690935410417e-05, P140=1.2028401454472503e-05, P141=1.4620340256727469e-05, P142=1.4717618562428873e-05, P143=1.053027710326371e-05, P144=6.3876375416827655e-06, P145=5.13452268362851e-06, P146=5.44115418023252e-06, P147=9.267250832939061e-06, P148=9.189666077136454e-06, P149=5.565113878695095e-06, P150=5.020608078880705e-06, P151=4.4288695935524825e-06, P152=4.06711735538213e-06, P153=4.367231797701346e-06, P154=3.7801993630093594e-06, P155=4.1437313002994146e-06, P156=5.793870216010709e-06, P157=3.829804816415911e-06, P158=3.2774432109581607e-06, P159=1.8386315719520758e-06, P160=2.0392073973773828e-06, P161=2.7397058270822285e-06, P162=3.2932216034331085e-06, P163=3.002692420746626e-06, P164=2.2763909755385693e-06, P165=1.3460508854639265e-06, P166=7.823484742131107e-07, P167=1.3244687451444234e-06, P168=2.4416988826394737e-06, P169=3.5979858868771017e-06, P170=2.897513095050454e-06, P171=6.374862119980205e-06, P172=3.7481540756908667e-06, P173=2.578172722464066e-06, P174=8.166486185796683e-06, P175=1.2325927060231217e-05, P176=1.1855326077831763e-05, P177=7.3591566708498006e-06, P178=2.9287937715031517e-06, P179=2.7226401567006096e-07),Row( P0=-465.4095617245511, P1=9.12927228790994, P2=-29.994786857746487, P3=-5.71048941088787, P4=-13.408135711515502, P5=-10.554468973549934, P6=-4.214642017752224, P7=3.757385938758665, P8=11.000899013313074, P9=13.078937605445306, P10=3.688353547812359, P11=4.652687089562315, P12=-8.752576026910786, P13=-4.444931487246359, P14=3.671958117441804, P15=5.731327384830534, P16=0.9723479444183474, P17=2.7785632411659416, P18=-0.4907177622893737, P19=0.7472589664285593, P20=-0.731791439300437, P21=0.6049289245663768, P22=-0.805987968143826, P23=3.164937614316012, P24=0.5856211910243787, P25=-1.6142285961410219, P26=-3.104570741627848, P27=2.5246378631320123, P28=-0.3079323742907191, P29=0.6983429530008276, P30=-1.5731845341532165, P31=2.223322206178778, P32=-1.1361837369227938, P33=0.3291588905604939, P34=-0.8475895757121366, P35=-0.5796623134297528, P36=-0.4667426808285335, P37=2.5301264859283705, P38=-0.2758134947541401, P39=0.647655856901584, P40=0.7206345215798848, P41=0.6757371142441811, P42=0.627498550033538, P43=0.5310517774723652, P44=0.4940789113418786, P45=0.5117116569823879, P46=0.5291991456110928, P47=0.5804706010427587, P48=0.5994659669496925, P49=0.6493169478632406, P50=0.679600265432499, P51=0.7035358896805463, P52=0.010666061755639226, P53=0.010555714028182263, P54=0.008171692235843764, P55=0.004813308214665026, P56=0.001516183245394059, P57=0.0018495327795376676, P58=0.0014542651858841834, P59=0.0008628949988319096, P60=0.0005596249536687249, P61=0.00025886489085772394, P62=0.0002229082922261577, P63=0.00015054779458798483, P64=0.00029668191321427647, P65=0.00037469444409877746, P66=0.00030846831419776384, P67=0.00029798034628032504, P68=0.0017349712684377404, P69=0.009748206643400694, P70=0.0142091579550609, P71=0.02083614468930509, P72=0.06501717463246973, P73=0.06360211388472573, P74=0.08074883949599693, P75=0.10131674694952615, P76=0.10283428790597154, P77=0.2753751643454693, P78=0.5063410253582318, P79=0.5756202904894836, P80=0.6607995974421736, P81=0.7995907852509019, P82=0.7823983833458612, P83=0.814521894863737, P84=1.2865073025002574, P85=1.055638067759996, P86=0.42043520653024335, P87=0.3493595227027314, P88=0.09700896609376997, P89=0.016647661911558125, P90=0.003908454711073042, P91=0.0041141222761333805, P92=0.005303775955946242, P93=0.0021270113110564, P94=0.0025129683703236034, P95=0.00369455107828383, P96=0.009563481100660244, P97=0.059746277741635295, P98=0.13403485152764644, P99=0.13038915555284308, P100=0.21118071785794296, P101=0.22066641603421577, P102=0.1345333867584665, P103=0.1444379143446491, P104=0.2513786957768842, P105=0.6307259169223625, P106=0.9804097820332597, P107=3.3133229954359114, P108=12.162557009297078, P109=1.7941340796755991, P110=0.12580736890864935, P111=0.04456899093140578, P112=0.0231348369766039, P113=0.01604880706861724, P114=0.021361910372567766, P115=0.02205557550893852, P116=0.04912598252283254, P117=0.1752849981188409, P118=0.2663193134648398, P119=0.31553745131230065, P120=0.5340233562580291, P121=0.148766146078278, P122=0.12635810829102379, P123=0.022795958265029767, P124=0.047024619199654095, P125=0.10739127002250412, P126=0.17476409189821593, P127=0.14016812969233786, P128=0.060157397783548956, P129=0.034885248646776994, P130=0.006153198778497804, P131=0.008873129599337655, P132=0.00794031344547743, P133=0.011742224725983274, P134=0.015670537258642755, P135=0.008236636410032826, P136=0.007025054880839818, P137=0.011085697769987529, P138=0.014954192270197675, P139=0.02201210804069008, P140=0.0069978497832753, P141=0.012952470073352205, P142=0.02334918501994111, P143=0.01058780039584605, P144=0.007137583608883549, P145=0.0037483772810523804, P146=0.0033577939850806867, P147=0.002740151977677572, P148=0.002726865168407121, P149=0.0018456754153699298, P150=0.0035675752150953056, P151=0.0024851453509360114, P152=0.0010571696674340201, P153=0.0009900427976223115, P154=0.0019216319335452739, P155=0.001347092819500435, P156=0.0011946485084071829, P157=0.0017277335964530891, P158=0.0012523292625674421, P159=0.001983917661915814, P160=0.002264799300535244, P161=0.0009350594703476726, P162=0.0014088571496976066, P163=0.0019550579165554132, P164=0.0009114524321993583, P165=0.000983186017965017, P166=0.00042151621108214404, P167=0.0011209460125347422, P168=0.0010018925419209134, P169=0.0007628426533743241, P170=0.0007796658020893734, P171=0.0004339310658613257, P172=0.00035343892367743394, P173=0.0003867852333183883, P174=0.0004507272814141172, P175=0.0005511812636085613, P176=0.0003246410716568386, P177=0.00024187436002110448, P178=0.00013646502244236256, P179=1.0293247808741627e-05),Row( P0=-556.5250261000932, P1=15.802378537012837, P2=-26.487255781317465, P3=-10.979623409627628, P4=-13.984138071290143, P5=-15.313992603530359, P6=-9.935807007169991, P7=-5.441409720092955, P8=0.04062434243590335, P9=0.4484448584201354, P10=1.1227856127848197, P11=16.920054275340956, P12=8.743410608823716, P13=-0.3181307582008139, P14=-1.161295353102718, P15=-9.478868756992444, P16=-5.70273991656128, P17=1.1167229634410427, P18=1.611718758991181, P19=0.5259200545205607, P20=-2.9536462326292763, P21=0.6280060745837373, P22=0.8973449359245143, P23=1.7919295555701775, P24=-3.201931169151127, P25=-3.592621272688353, P26=-3.4125188921931175, P27=1.5892908715449865, P28=-2.2736607409075122, P29=0.48928939077366795, P30=-0.7661003402012547, P31=1.0979989899021738, P32=-2.1780571684201635, P33=0.7165701916838358, P34=-1.7885034496390642, P35=0.5661005031302647, P36=-2.2168553840446594, P37=-2.2427655179308226, P38=-1.6712643845466162, P39=1.2383918622290904, P40=0.5706725149026399, P41=0.544450538859224, P42=0.5581574477058588, P43=0.5854112327676028, P44=0.6107212912195203, P45=0.6124022685478814, P46=0.6738691182839041, P47=0.7499355042097837, P48=0.7393335450324601, P49=0.6951754355942446, P50=0.6466566174400065, P51=0.5940993660359479, P52=0.00022445110943029785, P53=0.0001345400230867936, P54=7.529805586824034e-05, P55=8.268848098372661e-05, P56=8.86488107363561e-05, P57=0.00011912870879110539, P58=7.58366857971511e-05, P59=5.231280122237298e-05, P60=0.00010532579164775208, P61=0.00013207982464272648, P62=0.00011939571506692945, P63=9.766786593898294e-05, P64=7.108502361790623e-05, P65=0.00013287852674736698, P66=0.00040494426551588875, P67=0.0008402362855659238, P68=0.0035426267065627806, P69=0.014134494574409242, P70=0.015385952288056804, P71=0.026132681842958166, P72=0.06886803071937676, P73=0.09708090290558657, P74=0.32395503357491895, P75=0.6815507878271561, P76=0.6057448781553114, P77=0.39186783177188966, P78=0.25816152423577465, P79=0.3178745190201965, P80=0.22117278631145748, P81=0.0723120382023878, P82=0.01868494843303852, P83=0.006234287733543168, P84=0.004412893601158115, P85=0.0029025363781504604, P86=0.0010501579418590232, P87=0.0010304646328715761, P88=0.0011209944829967701, P89=0.0027606083548617974, P90=0.040052105873103365, P91=0.18095788676544036, P92=0.1155324994244618, P93=0.10761338457881398, P94=0.3844707274363215, P95=0.5174691165127879, P96=0.4609927790443582, P97=0.2799828320878557, P98=0.26689546358059135, P99=0.12843441654449692, P100=0.23331842378861894, P101=0.111674937603411, P102=0.010170304420708843, P103=0.02421977351043609, P104=0.04157119747543303, P105=0.03086249973388319, P106=0.03576240472250532, P107=0.02510623211890689, P108=0.007701659269611423, P109=0.014471502593094262, P110=0.041661537163675565, P111=0.0960300883392408, P112=0.14538499181776016, P113=0.03418024442007141, P114=0.029806673004136475, P115=0.12198193028777711, P116=0.10480660238323272, P117=0.10910651276480536, P118=0.11358037063175547, P119=0.017572110485891048, P120=0.015432543654149483, P121=0.01731239912992058, P122=0.01207816928609043, P123=0.022582989161716128, P124=0.025802739943921727, P125=0.00536218745470658, P126=0.0034552955449615526, P127=0.0023996384114439292, P128=0.0015600518498553413, P129=0.00857495666908086, P130=0.008110914147048183, P131=0.0015568363875204207, P132=0.0015120924432966202, P133=0.0010152286296406016, P134=0.001134004245751607, P135=0.0010354955146044426, P136=0.0008277176602578992, P137=0.0032148622666207153, P138=0.005474669430757255, P139=0.004787987272646697, P140=0.0014347750305818679, P141=0.0009979486388987846, P142=0.001750510156350094, P143=0.0027328323614652238, P144=0.001352607839147685, P145=0.0015448909601613306, P146=0.0015401266243572542, P147=0.0012954641414573873, P148=0.0012114239962118126, P149=0.0007527878578971262, P150=0.0004147367780596542, P151=0.000276990895019446, P152=0.00018020627691063453, P153=0.00012900036798776989, P154=0.00012912400479536432, P155=0.00023293119780278757, P156=0.0002593833911134969, P157=0.0002761798483951228, P158=0.00030255663743725505, P159=0.00035027669848975617, P160=0.0004062886248300796, P161=0.00042273364942256825, P162=0.0005069131653730122, P163=0.0004349770090240905, P164=0.00037077438665423357, P165=0.0005591460147557158, P166=0.0005509705326224377, P167=0.000346991814296552, P168=0.0002855485351283777, P169=0.0002937155447510303, P170=0.0003661546551140587, P171=0.0004670048245519523, P172=0.00048808506537725295, P173=0.0006011238113360134, P174=0.0007262911077370525, P175=0.0006477168517472711, P176=0.0009083452258665209, P177=0.0013647486543212118, P178=0.0007598167944001352, P179=6.48127527139664e-05),Row( P0=-621.4692321828039, P1=30.765585351318805, P2=-24.648378891544855, P3=-7.6841493602738815, P4=-11.341376081121258, P5=-13.881656286807443, P6=-7.96290355435058, P7=-9.195485202828698, P8=-3.537440272794357, P9=-8.010173738794187, P10=-9.583932944153242, P11=-0.5302610017783144, P12=7.7794270278595805, P13=6.971505842447664, P14=15.668251046440355, P15=6.05472319883217, P16=1.8130566681661417, P17=-5.365014725141963, P18=-4.579583642870384, P19=-1.103025085557739, P20=-3.900363475922739, P21=0.7859898039404208, P22=-2.8059112556106593, P23=-4.113038008201833, P24=-2.7979152359177863, P25=-1.3395759246043721, P26=-1.4535341224126275, P27=2.782787635475255, P28=-2.9684730224522333, P29=1.1578994340140298, P30=-0.8952794919615077, P31=0.5399195322499737, P32=-0.6355073592847946, P33=0.9960561659052326, P34=-1.736295231450013, P35=-1.3250108488721166, P36=-3.4380430681928695, P37=0.9761362106232504, P38=-1.1056948593020703, P39=-0.31277076569113843, P40=0.6750606610468297, P41=0.7310167851842995, P42=0.7696391493794251, P43=0.7468615416097891, P44=0.7529843432686463, P45=0.6827730166312284, P46=0.6187561910051855, P47=0.6145615120315946, P48=0.6564224986175077, P49=0.6053530562280298, P50=0.578028104276371, P51=0.6223549760822497, P52=3.081481824884044e-05, P53=2.1017441275393923e-05, P54=3.517124988016147e-05, P55=4.613568516718042e-05, P56=5.2972990125704875e-05, P57=8.435506307842012e-05, P58=5.373820174573818e-05, P59=3.237489220942208e-05, P60=2.939083543479276e-05, P61=3.1347972027196265e-05, P62=5.58686445576406e-05, P63=8.922619201937942e-05, P64=0.00040878255919211075, P65=0.003371795764486026, P66=0.012075993071891044, P67=0.022084491901575475, P68=0.05485074223336119, P69=0.16169363581732138, P70=0.1458255602112845, P71=0.10559691344891872, P72=0.15582195948745697, P73=0.17728956535004659, P74=0.16250661594975327, P75=0.05677197944534528, P76=0.01305062358961762, P77=0.006006703637816987, P78=0.003553668749512959, P79=0.001772804846931151, P80=0.0008107663001771055, P81=0.0007141689445254537, P82=0.0007290785273381782, P83=0.0020124162745769622, P84=0.009551758725341754, P85=0.05118367457815222, P86=0.11115861115864382, P87=0.1408771962886227, P88=0.2550636781635027, P89=0.24608547967348582, P90=0.16208688752269984, P91=0.07374916266729818, P92=0.035707916379262794, P93=0.013622896279945612, P94=0.007391606809702788, P95=0.0019661127045360913, P96=0.0005173563158198009, P97=0.009455350831108787, P98=0.031204024667815532, P99=0.02304809709108846, P100=0.018013033829494905, P101=0.00872860687398559, P102=0.00491015273311488, P103=0.033648484040872756, P104=0.045488785358267746, P105=0.030492877523509924, P106=0.009966910975804844, P107=0.002844811810632793, P108=0.0032848558897564353, P109=0.013957168362702953, P110=0.009688376483797475, P111=0.01880462115536261, P112=0.04412146545622803, P113=0.011541298570267828, P114=0.005423130097809334, P115=0.0075344252029278915, P116=0.001952744799584261, P117=0.0022447728013122543, P118=0.005157986897178246, P119=0.022362341683038064, P120=0.01083435068566627, P121=0.0025408184338405553, P122=0.0013100708528532177, P123=0.0011778295164455014, P124=0.0030655981988972243, P125=0.0016320785389186184, P126=0.0003754585599104753, P127=0.000523160051554636, P128=0.0008189638684652928, P129=0.000753639984806631, P130=0.0001747747574286419, P131=0.00022454208035869376, P132=0.00039003685076857675, P133=0.00013846910756170713, P134=0.00012402887101277787, P135=0.00028436931946116583, P136=0.0002927021968431337, P137=0.00032146200442452016, P138=0.0004285664163091724, P139=0.0003515353289891521, P140=0.0002506922658627234, P141=0.00029203344841734327, P142=0.0003988999730258161, P143=0.0002838350812421477, P144=0.00024516437154671497, P145=0.00019644077148366423, P146=0.0002160777257878984, P147=0.00032343717151498455, P148=0.00023668906454617306, P149=0.00021847731010608715, P150=0.00012423802550352918, P151=0.00011569690466606504, P152=4.4892502590829546e-05, P153=5.1594302121652526e-05, P154=9.08933254031846e-05, P155=0.0001248313220648865, P156=0.00021840379017199865, P157=0.0002062528233426182, P158=0.0001832634139208751, P159=0.00029570768606274696, P160=0.0001749857702603087, P161=0.00022008068137942095, P162=0.00038167732972556627, P163=0.00021324453137125802, P164=8.65693640504027e-05, P165=5.501689795558134e-05, P166=5.2986015167183106e-05, P167=4.2311255266179805e-05, P168=4.0565290329878474e-05, P169=3.969664319936024e-05, P170=5.317176856333436e-05, P171=6.454183669453687e-05, P172=5.707559026942733e-05, P173=6.312824439573188e-05, P174=7.645500182820177e-05, P175=0.00010639474861056171, P176=0.00011390028686096322, P177=0.00011851644343184908, P178=8.33790168609668e-05, P179=8.088702673750918e-06),Row( P0=-514.3517203350085, P1=9.771488696225784, P2=-38.11230226198904, P3=-10.824939782715195, P4=-15.138908012836097, P5=-15.358413821086785, P6=-8.361352639538286, P7=-4.6411006168019, P8=3.4722518910793005, P9=7.340014015543503, P10=-0.2231503627679031, P11=16.46144746760492, P12=5.9016656520061215, P13=-6.584650758493559, P14=-1.6390417329482716, P15=-8.582650638182848, P16=-3.8525729957469137, P17=1.9272730984984396, P18=1.0095486847314, P19=0.24086105929258164, P20=-2.3272159830824424, P21=1.0784293886339025, P22=1.9998612006295473, P23=0.5821697033737612, P24=0.43834574876203314, P25=-2.055652853346996, P26=-3.653245929012887, P27=3.2243807910218587, P28=-4.437298354290225, P29=0.39518719774754274, P30=-0.8388988656035257, P31=1.2254050422304001, P32=0.5513100084738497, P33=-0.0964138133051067, P34=-2.084105989754184, P35=0.2983303270626885, P36=-3.397285025183589, P37=-1.5443987128215275, P38=-0.5653676613760237, P39=2.934324419499804, P40=0.6392032981989016, P41=0.5457154973630235, P42=0.5567570251586654, P43=0.5552929952957169, P44=0.5805835855514961, P45=0.6417478135285477, P46=0.7326400962938112, P47=0.7858308036571463, P48=0.7898289089684827, P49=0.7185529023305378, P50=0.6704579352606975, P51=0.6636903075083043, P52=9.260435309561588e-05, P53=7.15969885743342e-05, P54=7.017309607389647e-05, P55=6.322405952064132e-05, P56=7.744057454109104e-05, P57=7.303327867155258e-05, P58=4.9016999035851204e-05, P59=5.5293985273705336e-05, P60=6.848785194742172e-05, P61=8.321884516483326e-05, P62=8.652999036798608e-05, P63=7.987559238194241e-05, P64=6.159343731987461e-05, P65=0.0001387644322254432, P66=0.00022597451932356732, P67=0.00036053485162297063, P68=0.0012726677904526277, P69=0.0028355911622912327, P70=0.00324233804418117, P71=0.00941059742116527, P72=0.030613653643591105, P73=0.04246940157492807, P74=0.13870668318303356, P75=0.3204165477621514, P76=0.5895792283192042, P77=0.6511913219728199, P78=0.4712304611135652, P79=0.3992117940031878, P80=0.3374407219342936, P81=0.28782222366770616, P82=0.4048060658816946, P83=0.5532971737115547, P84=0.6782013947991273, P85=0.5203636226776679, P86=0.09335771957435207, P87=0.018720783341113455, P88=0.009221999932542925, P89=0.005169583800494191, P90=0.008232132861023164, P91=0.02069200490992086, P92=0.08362012371713232, P93=0.1548676136248243, P94=0.827406494739111, P95=1.02199763885861, P96=0.5189086650559113, P97=0.6676899408232018, P98=0.5716246460622675, P99=0.5639626865573896, P100=0.5121728592071416, P101=0.10457723663720946, P102=0.062440142327591056, P103=0.050636866501189176, P104=0.06155694543145689, P105=0.08874070842356792, P106=0.0629979961377334, P107=0.03561304572001284, P108=0.03530367712195792, P109=0.12193185064457693, P110=0.36586164335694327, P111=0.4221939324698187, P112=0.15677130534681374, P113=0.14143943860794583, P114=0.21709896039495977, P115=0.2709085536308431, P116=0.3034772975176412, P117=0.10118102903485815, P118=0.04077304654437287, P119=0.03124546607754125, P120=0.09051911599909641, P121=0.09293093334010125, P122=0.03276079812104227, P123=0.043347418307276765, P124=0.03826489097703761, P125=0.03248440591896378, P126=0.0212489493845561, P127=0.03097467186164192, P128=0.04538047621365291, P129=0.03834565968193985, P130=0.012107590543799228, P131=0.008876627343572661, P132=0.007242130632963765, P133=0.004715361462285816, P134=0.005130578773574968, P135=0.004127871727836373, P136=0.004213490563868309, P137=0.014906263177680578, P138=0.01548125872012418, P139=0.010718439968333559, P140=0.009677123456852621, P141=0.0077003366653283205, P142=0.004028911604644559, P143=0.005355588967137705, P144=0.009478764962077738, P145=0.006803089953124353, P146=0.00848957735675628, P147=0.007689097778578091, P148=0.0019010974171261655, P149=0.0010658159419259655, P150=0.0004471491550218479, P151=0.0003796422742778325, P152=0.00045545089550653214, P153=0.00019244683434958438, P154=0.00022013622730233777, P155=0.0004655061966013325, P156=0.0006100892594374176, P157=0.0006288411907624973, P158=0.0008338946490396977, P159=0.0008645374177436915, P160=0.0009620799341291968, P161=0.0010233002649262797, P162=0.0005386867699311262, P163=0.0005977704717209834, P164=0.000726099821475693, P165=0.0006281946004127645, P166=0.0004932072622012018, P167=0.0005047671127013375, P168=0.0004228267228206358, P169=0.0006215332951090365, P170=0.000828140645159791, P171=0.0006644113348322857, P172=0.0005375466419251516, P173=0.0005784746961691405, P174=0.0004886598684539732, P175=0.0003469987300818432, P176=0.00041580191231607445, P177=0.0003699761898970352, P178=0.00017988237610938493, P179=2.3933415276422055e-05),Row( P0=-624.0217053096762, P1=34.4818130059359, P2=-13.873639002676315, P3=-8.77752560202458, P4=-8.93109307096421, P5=-10.951689488451205, P6=-7.4831150342682395, P7=-8.654827810769943, P8=0.6946899518723368, P9=-8.836231435820212, P10=-11.837263259247528, P11=-2.886791942991993, P12=-2.95263805471733, P13=1.9487622594735314, P14=11.503357967832025, P15=10.372111500459678, P16=10.724514257145945, P17=4.030039166251591, P18=2.0058061421962456, P19=-0.6397053684910488, P20=-4.119672355513028, P21=0.16227715741787355, P22=-0.4708768970072669, P23=-1.9509293426106311, P24=-3.827002707054865, P25=-6.733655379760268, P26=-3.980419974030198, P27=0.5886835238652421, P28=-0.8237631536754536, P29=1.3125846568470474, P30=-1.9072085352476964, P31=1.0011374798460968, P32=-0.6477407457378467, P33=0.2715047833534733, P34=-1.4681148827202661, P35=-1.3166153654867936, P36=-3.1436408768363466, P37=0.524232171524997, P38=-1.4336066675851797, P39=1.9270208978929064, P40=0.7128632066986784, P41=0.762016065009579, P42=0.7284159387063062, P43=0.6829239206880252, P44=0.6192824832207829, P45=0.5868454191913537, P46=0.5696634732045674, P47=0.5705691394783713, P48=0.6062660305895743, P49=0.611429779063111, P50=0.6048215996943668, P51=0.6057508436148492, P52=0.0003379698057326421, P53=0.0004314501738475378, P54=0.0002718528287374587, P55=0.00010026511150126252, P56=7.619353117560984e-05, P57=7.828893781571376e-05, P58=4.662321146997063e-05, P59=3.164800549598365e-05, P60=3.040641040560456e-05, P61=4.544638045831087e-05, P62=0.00021379570687252093, P63=0.0010985817586971517, P64=0.0039578100292314115, P65=0.023784882350300664, P66=0.056255340168111574, P67=0.09762238162585604, P68=0.17693981532982342, P69=0.30853102036188157, P70=0.23845841487533098, P71=0.12642312188366897, P72=0.0313573217732571, P73=0.008157002698244407, P74=0.0014370829685310174, P75=0.0008626832150863157, P76=0.0004133832529798441, P77=0.00038562084964503877, P78=0.001704347015707465, P79=0.007243674202361546, P80=0.007973767640057248, P81=0.006812578321385556, P82=0.019007580972652476, P83=0.07703367766493194, P84=0.16695561270718778, P85=0.2461227986885098, P86=0.08404511822441478, P87=0.0313093337655576, P88=0.04455914197369297, P89=0.03953478351322401, P90=0.024889923135948185, P91=0.016991042213437058, P92=0.014296244892355408, P93=0.007675455344706594, P94=0.011371713904603703, P95=0.009952847952543097, P96=0.005734180705383091, P97=0.012713071957859893, P98=0.015938794534748103, P99=0.006923294747782413, P100=0.010879575310791607, P101=0.008197320032407583, P102=0.001093662744530612, P103=0.0006008008534289495, P104=0.0017021940911012953, P105=0.005139767402692137, P106=0.0033832017244129617, P107=0.0014103910543313803, P108=0.000944192250180399, P109=0.0017392452430335162, P110=0.002314353797989332, P111=0.0017416288183653146, P112=0.0012077229236215543, P113=0.00099881110158303, P114=0.0019088913470567308, P115=0.0035375416446335062, P116=0.0030464129410255757, P117=0.002131591180994442, P118=0.0011308564287051738, P119=0.0010836993191612907, P120=0.0008589683767304627, P121=0.0010651440872424154, P122=0.0009848774005779756, P123=0.0004981472211076324, P124=0.0007075241962157682, P125=0.0002575303950345972, P126=0.0001243832448216923, P127=0.00021609862265024898, P128=0.0002761979110401457, P129=0.00017322836820940274, P130=4.201762160429922e-05, P131=8.918134833882001e-05, P132=0.00014478537351795135, P133=0.0001655135039019214, P134=6.536343748106658e-05, P135=5.459071629129549e-05, P136=8.159575890263645e-05, P137=7.672093352247934e-05, P138=0.00010689769427744062, P139=8.65204713189635e-05, P140=0.00010051143612997082, P141=0.00013098442028109097, P142=8.674617433017054e-05, P143=6.78593916523426e-05, P144=5.892794082772365e-05, P145=6.772468655189409e-05, P146=0.00010063320977106565, P147=0.00011232508743057548, P148=9.878855555047019e-05, P149=9.067098719707825e-05, P150=6.263873946672066e-05, P151=4.201623613319255e-05, P152=3.8537187359714644e-05, P153=1.800885203421603e-05, P154=2.259355160248369e-05, P155=5.6299865396547274e-05, P156=8.689673578320193e-05, P157=0.00011208468547744644, P158=0.00013020292584453336, P159=9.523177450088724e-05, P160=0.00010477895474297089, P161=0.00013068607809615712, P162=0.00015109212640200062, P163=0.00015654567699113854, P164=0.00022002627379571853, P165=0.00018352186001379417, P166=0.0001988391918627485, P167=8.700544505341874e-05, P168=0.00017618634470536562, P169=0.00017192992263399863, P170=0.00016455411726872644, P171=0.00022307833940746975, P172=0.00013556693648369256, P173=0.00014658709037386198, P174=0.00022991019777755662, P175=0.0003612347310530183, P176=0.00030322652144055857, P177=0.0002625126448187418, P178=8.935254417369853e-05, P179=1.1131756764535201e-05),Row( P0=-706.4328230727162, P1=50.41799131809193, P2=-4.066739051884725, P3=-6.395596641498534, P4=-10.563986350632662, P5=-11.972240531424658, P6=-10.459303985027283, P7=-8.516837396694221, P8=-2.737827468019768, P9=-12.304434909426066, P10=-9.861888978720591, P11=-10.674613324635422, P12=-10.35311125297971, P13=-9.555832778633425, P14=-7.00203467183901, P15=-1.2719938912613717, P16=-2.3928322821561685, P17=4.2913254060870125, P18=5.165712024805193, P19=5.260094785935968, P20=4.8820235577087505, P21=2.957931180346424, P22=2.6567102324767005, P23=4.6533585516834295, P24=2.491429864950691, P25=1.4246338037880784, P26=-1.8873027750620859, P27=-2.2651160750667785, P28=-4.1922313721261535, P29=-0.5048989396915592, P30=-0.8805102767188149, P31=-1.2104578086231994, P32=-2.823693465323858, P33=-0.9989166787358233, P34=-0.581347236485321, P35=2.514710637418556, P36=-2.992258800403323, P37=-1.7327456477906855, P38=-4.883129636624211, P39=-0.14035439448841835, P40=0.5857576597214041, P41=0.5255466051814301, P42=0.5471593514215871, P43=0.5600417773654787, P44=0.6038020331750952, P45=0.6088346530033696, P46=0.6280242606879854, P47=0.6424145390571295, P48=0.6030149092247862, P49=0.567917019551544, P50=0.601949722460404, P51=0.6353526853811243, P52=6.772545055561636e-07, P53=2.659305763902765e-06, P54=7.234932414912957e-06, P55=2.2351180588562686e-05, P56=4.305071965319272e-05, P57=5.056964436179236e-05, P58=3.9725602289420726e-05, P59=5.056524139749868e-05, P60=0.0011617758039494247, P61=0.01079738356003412, P62=0.03291691401212358, P63=0.058316783280381396, P64=0.0378293103085154, P65=0.05853969069639902, P66=0.11001357636933184, P67=0.0783123079078932, P68=0.015817702587134868, P69=0.0007966995350037465, P70=0.00025346677338415385, P71=0.0007299588264424383, P72=0.0014111900488919876, P73=0.0014159151179122613, P74=0.006873695833211905, P75=0.014649448742758078, P76=0.010072113971323996, P77=0.00486108424653418, P78=0.0026074820084739563, P79=0.0036306992671306898, P80=0.011490691727093699, P81=0.01944146370675719, P82=0.017036552500027228, P83=0.008087533363347287, P84=0.005558582710050836, P85=0.004232356983876374, P86=0.006418406085962219, P87=0.00570292872218352, P88=0.006078941852721185, P89=0.0020106357529040353, P90=0.001301261787159317, P91=0.002094853859973771, P92=0.001212773309050869, P93=0.002415054577931128, P94=0.0053331574402948004, P95=0.006534107143576251, P96=0.0025463603663100913, P97=0.0006657504826452009, P98=0.00018643076861971536, P99=8.311816083420041e-05, P100=7.856921589771983e-05, P101=9.279999014928996e-05, P102=0.0020654308389308025, P103=0.002495592471275687, P104=0.0023003805929282307, P105=0.001984886706587498, P106=0.0003495641276024484, P107=2.5621032916199318e-05, P108=0.00011214180597552186, P109=0.004363474676819029, P110=0.0028918251746541074, P111=0.0006210406607640097, P112=0.0002041496195210825, P113=4.853755007122049e-05, P114=0.0002312050558536604, P115=0.0007507064622042286, P116=0.00023620628758411278, P117=0.00017719705485734667, P118=0.00017351107741033465, P119=0.0004326748615166144, P120=0.0006775893600537392, P121=0.00035195724356612795, P122=0.00012804553148077773, P123=0.0001931286867775979, P124=0.00028130927881417074, P125=6.726424717127363e-05, P126=4.950978555165126e-05, P127=5.772971777084425e-05, P128=2.140224838818201e-05, P129=2.2658518104334384e-05, P130=1.3897294472320862e-05, P131=9.143899189400727e-06, P132=5.258107463536771e-06, P133=3.852464785801311e-06, P134=3.714747673521607e-06, P135=5.119148686011755e-06, P136=9.436488451168842e-06, P137=6.22963147661222e-06, P138=1.3285253078188984e-05, P139=1.8752267691554537e-05, P140=2.3582808935853546e-05, P141=1.8381786291371957e-05, P142=6.957516781343259e-06, P143=5.654772390093528e-06, P144=5.033994298244199e-06, P145=5.896823380091765e-06, P146=4.751293503477039e-06, P147=5.543879601522636e-06, P148=5.875207318599438e-06, P149=1.0056214473860284e-05, P150=1.0560542617518672e-05, P151=1.1861706810193284e-05, P152=5.8046950676652475e-06, P153=7.807190271389234e-06, P154=1.058188217118133e-05, P155=2.3946101844685416e-05, P156=1.9133703005385907e-05, P157=1.1409522840929246e-05, P158=1.1988837102662938e-05, P159=1.1296748598853567e-05, P160=1.8209706499169112e-05, P161=2.383655857090091e-05, P162=4.0325339842980774e-05, P163=2.9176067764983593e-05, P164=4.861531785502002e-05, P165=3.731664331285283e-05, P166=2.272779991206682e-05, P167=1.647982821203643e-05, P168=2.872753169006047e-05, P169=2.3651651507494032e-05, P170=2.2905179417778355e-05, P171=3.458374615690863e-05, P172=4.2093578030606566e-05, P173=2.956680388236597e-05, P174=1.8901881490187786e-05, P175=2.0268333966916174e-05, P176=1.764333320982512e-05, P177=4.133744390816013e-05, P178=1.90855956843621e-05, P179=1.0755827148073324e-06),Row( P0=-517.5845580210809, P1=20.687366753001374, P2=-32.12929990238437, P3=-12.53557009103035, P4=-15.673722195895921, P5=-11.256743627922496, P6=-9.939713105053347, P7=-10.89346108629172, P8=-3.3693890632279517, P9=-11.722375687318177, P10=-11.49394286270691, P11=3.2680402094253425, P12=2.32052032515323, P13=4.975676446994218, P14=5.043210769800399, P15=0.2928259206741798, P16=-2.3187512188234103, P17=-2.5529937343545495, P18=0.06683546363079318, P19=-2.015738458153829, P20=-2.2655950200664785, P21=-0.8141347747006443, P22=-0.12224567269417357, P23=2.4675388848492643, P24=1.0220154285441176, P25=-2.6536929944747296, P26=-1.7149598537907762, P27=0.3198766862117706, P28=-0.8979263920803655, P29=1.2842907629173896, P30=-0.986178006131574, P31=0.6724687715746345, P32=-0.15615658729839296, P33=0.4074705238072734, P34=-2.5760491296281827, P35=-1.0979278274762703, P36=-2.458680128424761, P37=-1.0640114214119598, P38=-2.4052588707889844, P39=0.7757034254607141, P40=0.6629839129595686, P41=0.6750609763989971, P42=0.6744255406706816, P43=0.6735578766147506, P44=0.6923443904948079, P45=0.6931888623661258, P46=0.6784133888492927, P47=0.6675402534706268, P48=0.6281936187400177, P49=0.5874538293805208, P50=0.5867475329039258, P51=0.5999244857540766, P52=2.1217922682742147e-06, P53=2.6707643336692577e-06, P54=1.0835639288529222e-05, P55=0.00010846945705468735, P56=0.0003547690167616368, P57=0.00033251699999936867, P58=7.51267044113779e-05, P59=0.00012212703720388, P60=0.0009773048093173493, P61=0.002025774576957332, P62=0.001961480553829527, P63=0.0015535488974955205, P64=0.0027006224614053794, P65=0.012071208807740829, P66=0.027602733284972, P67=0.03741525143681079, P68=0.08533706693171922, P69=0.13481166818408907, P70=0.11816017960711143, P71=0.11341887402612574, P72=0.11815787564034555, P73=0.14075541441080663, P74=0.20101654862872498, P75=0.23241670568403763, P76=0.14471650794605978, P77=0.0223688756781926, P78=0.004773377921186861, P79=0.0036298538913584573, P80=0.004418140417342415, P81=0.005015396590599455, P82=0.008695783743989795, P83=0.024040105171467027, P84=0.084432758940384, P85=0.17936708300213638, P86=0.13418293937394407, P87=0.08249012508423109, P88=0.10223511812613557, P89=0.28325754974330836, P90=0.4151037118971431, P91=0.303515477334046, P92=0.21640093204840444, P93=0.5972463213938605, P94=1.4864742224745278, P95=1.074834145084802, P96=0.6940030058661679, P97=0.4640500747349165, P98=0.10178689534850129, P99=0.09803536500831107, P100=0.0911384664057408, P101=0.07235573868332634, P102=0.058425215743456785, P103=0.03412414935473582, P104=0.05524972172249431, P105=0.1580148949875738, P106=0.09565941952354322, P107=0.04616468963258342, P108=0.05062226848853391, P109=0.050903552965188545, P110=0.06579628540466062, P111=0.0548816651253413, P112=0.040773368275050854, P113=0.10494830149102066, P114=0.7383064206976777, P115=0.28819923594855695, P116=0.047848942426632124, P117=0.06173216986359115, P118=0.10754118006305614, P119=0.09728208376867757, P120=0.06746425968748243, P121=0.05474607024530962, P122=0.04096777092361891, P123=0.05090524406092989, P124=0.03815704114601088, P125=0.018344694292243023, P126=0.012504378176159861, P127=0.007612394049031576, P128=0.006996223954872157, P129=0.00450430303377394, P130=0.0026001511139673683, P131=0.006385929172159726, P132=0.0034549830130326644, P133=0.0017918488222100843, P134=0.0044320342430306655, P135=0.005405662687358723, P136=0.0037754645465052834, P137=0.0015396415550232457, P138=0.005850279036969792, P139=0.007069570130041737, P140=0.003025024821309307, P141=0.0037248545954356614, P142=0.01135071675550694, P143=0.0030602509496409657, P144=0.001953104235657817, P145=0.003288932605234928, P146=0.0022075290557367335, P147=0.0011901383505579623, P148=0.0016526527464893969, P149=0.000988801598993828, P150=0.0011172856063757038, P151=0.0007502924174491289, P152=0.0004958425760557777, P153=0.0002030772929210234, P154=0.00014774594563823185, P155=0.0003456142718035654, P156=0.00019383816519217347, P157=0.00013254390322648318, P158=0.0001589122885741528, P159=0.00017853272776594846, P160=0.00020997143474032527, P161=0.00018211980950403974, P162=0.0002544409043209887, P163=0.00033074840753649664, P164=0.00038897835445187947, P165=0.0004828502394374086, P166=0.0005857699503316672, P167=0.0004438838136409385, P168=0.00029377031192897684, P169=0.0003451769214586496, P170=0.0004197859261092234, P171=0.00044937255112411577, P172=0.0004367072502737801, P173=0.000625127417547745, P174=0.0006548752543959057, P175=0.0010152106373110717, P176=0.0016755077715161948, P177=0.0019976924955051717, P178=0.0016734297200995017, P179=0.0002479922538436098),Row( P0=-614.5202132764382, P1=39.02859901040337, P2=-25.713051756884553, P3=-14.724869042249095, P4=-13.212578419077563, P5=-12.955178769685594, P6=-13.825885154052404, P7=-6.036989997609513, P8=-4.287925511465675, P9=-11.709991859059278, P10=-14.84314635596541, P11=-4.442354451463542, P12=-3.4015511142514305, P13=-3.518603861938322, P14=3.0644086713404897, P15=1.4268680955671809, P16=2.0601835367517576, P17=-1.1043657563297742, P18=5.562034936067259, P19=4.1864150926611305, P20=3.024543692650925, P21=4.997634678191552, P22=-1.4265784372135926, P23=-1.3017408007822833, P24=-0.9659559837992363, P25=-2.8171929273393475, P26=-0.1704847000531886, P27=2.4754772974761283, P28=-2.489333354355712, P29=0.43723262918629346, P30=-3.9803089164940597, P31=1.6040404301990134, P32=-1.8541999475780924, P33=-0.6133756716631128, P34=-2.225948226040075, P35=-1.8283744829727222, P36=-1.685115506031987, P37=0.7264823099551302, P38=0.11352742359451712, P39=0.8985258065385747, P40=0.697515242996031, P41=0.6987564913076922, P42=0.7114121289936425, P43=0.7075240734740006, P44=0.7073429737026609, P45=0.6703041330090092, P46=0.625810150799792, P47=0.6500580847650456, P48=0.6652539650020107, P49=0.7016321876939756, P50=0.6677560783759491, P51=0.6389024373005068, P52=7.812078212598865e-06, P53=4.701096462465274e-06, P54=2.0262734388388365e-05, P55=3.4684507943624446e-05, P56=3.8941068616065885e-05, P57=5.154762666813676e-05, P58=0.00012450576220478056, P59=0.000835970984182444, P60=0.0044039946114867706, P61=0.008087807639216672, P62=0.01348765439225169, P63=0.025608646670651918, P64=0.07071864844805074, P65=0.1457690954850959, P66=0.09277865427765412, P67=0.053572638877288296, P68=0.042150290201579586, P69=0.05574734680773397, P70=0.061988965558902515, P71=0.0669090837265089, P72=0.03735469098414399, P73=0.031786698938279824, P74=0.020045269602754722, P75=0.014014412569509806, P76=0.018478829896616443, P77=0.045448629525115664, P78=0.11051654657169083, P79=0.10950593833830154, P80=0.09806043190369318, P81=0.05773799596403026, P82=0.0193056389226172, P83=0.011740545404688362, P84=0.01812772749005238, P85=0.02639765233411579, P86=0.023432672236427796, P87=0.04518372396337086, P88=0.10317540301112395, P89=0.11792471410405299, P90=0.14949826637363328, P91=0.19194855970799357, P92=0.10637299329767956, P93=0.04505001055845427, P94=0.02191083915410214, P95=0.013708279192983331, P96=0.007929627351772316, P97=0.01394984551539319, P98=0.012902062918818638, P99=0.015537708850858273, P100=0.036928206613024615, P101=0.023087982499317376, P102=0.01415461389256124, P103=0.007541941440993708, P104=0.014676750203586945, P105=0.05576418301144453, P106=0.034917467378590916, P107=0.007079157456637702, P108=0.0056586914777810885, P109=0.006363191121774638, P110=0.0038044038062159348, P111=0.0028536485055868912, P112=0.002949343040463678, P113=0.008447915419359358, P114=0.011972343442396585, P115=0.0060530428317474285, P116=0.0030759102135855844, P117=0.0036454685840852648, P118=0.0024085078829843482, P119=0.0012178947221428288, P120=0.0007498253819027163, P121=0.007031466314820535, P122=0.00469149634660741, P123=0.0014172950022592076, P124=0.0010131797631139554, P125=0.002584391656947376, P126=0.00135970547672451, P127=0.000558836164445411, P128=0.0007230067824384517, P129=0.00033751216740481873, P130=9.64256678474876e-05, P131=0.00015160652890724245, P132=0.0001325053991795851, P133=0.00012807283432644446, P134=0.00018921769491634063, P135=0.00016792624723249909, P136=0.00037758091403051526, P137=0.0003740589380378793, P138=0.0004866835694551266, P139=0.0005224988807302887, P140=0.0003354126467360774, P141=0.00016088585196247845, P142=0.00017346453829264065, P143=0.0001309979935090619, P144=0.00014121129101279833, P145=0.00012847872367204567, P146=6.437108909912766e-05, P147=6.844967782537247e-05, P148=6.920449193527067e-05, P149=0.00010549855105495843, P150=0.00012054269272712879, P151=6.436737270580456e-05, P152=4.312679048270324e-05, P153=2.2398491306059585e-05, P154=1.663223556457703e-05, P155=2.4108526155261813e-05, P156=4.345057785876865e-05, P157=7.758642396162745e-05, P158=9.020576833443204e-05, P159=0.00010107219731051371, P160=0.00017644999425113882, P161=0.00024316944468520354, P162=0.000265798873798104, P163=0.00045741259861949797, P164=0.0005712771731631404, P165=0.0008294694575583108, P166=0.0009016311591679186, P167=0.0007036725850343259, P168=0.00036209845458736785, P169=0.0003433712538138753, P170=0.0004067609271038897, P171=0.0004969115352622489, P172=0.0006885261695150142, P173=0.0008146666574622618, P174=0.0009864908941079173, P175=0.0009604938887270162, P176=0.0008232411948511266, P177=0.0008621063824068834, P178=0.0006178169293021126, P179=4.128485379453461e-05),Row( P0=-524.6004175207746, P1=16.547349927658793, P2=-30.378763413090326, P3=-10.15885570895308, P4=-20.16757279285933, P5=-10.807688970507323, P6=-13.721838089223501, P7=-11.797535489947895, P8=-5.745881172294781, P9=-8.59989770636582, P10=-9.714646559427276, P11=6.144362065673558, P12=9.025899165227557, P13=7.991719205108686, P14=8.871427160204682, P15=1.8041475652065793, P16=-3.168859304894597, P17=-4.422696977977483, P18=-1.3215302012894523, P19=-0.3709323662912923, P20=-0.6523378985871641, P21=0.44895458642471553, P22=0.018350546522194727, P23=0.9352493302424758, P24=1.148881160464505, P25=-0.502663935600673, P26=0.2746398172502329, P27=0.5368322560515132, P28=-0.24445891127583239, P29=-0.18503595104844311, P30=-0.7008004264204827, P31=2.710349990868553, P32=-0.1952254042100617, P33=-0.39892313629995857, P34=-0.9233011405789014, P35=2.2408367987691835, P36=-1.4917657427063329, P37=-2.44040423089106, P38=-2.4485432698259575, P39=0.8118570839345545, P40=0.5812264101276312, P41=0.6052104215976769, P42=0.6812541332830394, P43=0.6617973038929428, P44=0.6712220599826059, P45=0.6398952877475398, P46=0.6500779975840614, P47=0.6960430041646731, P48=0.657532297543066, P49=0.6041920867430749, P50=0.5932443962024874, P51=0.5923761627391975, P52=8.752325954498356e-07, P53=3.419369205290897e-06, P54=4.179288144698997e-06, P55=2.7932693546447924e-06, P56=3.214278652166572e-06, P57=4.570517743272663e-06, P58=4.488608063500344e-06, P59=1.0698206492674664e-05, P60=5.639880910282232e-05, P61=0.0003409287906394228, P62=0.002404077464993304, P63=0.005032161098013015, P64=0.004365754654176276, P65=0.00826646037675743, P66=0.015519753095339414, P67=0.01747558575375831, P68=0.02756018536019451, P69=0.08439155986741592, P70=0.10453886486801171, P71=0.09404711255862697, P72=0.10112920668435847, P73=0.09083286690546144, P74=0.18954160258607536, P75=0.3902774910986535, P76=0.27424693340825446, P77=0.05537216916888865, P78=0.004621635224406114, P79=0.001175522739859724, P80=0.001125192121162922, P81=0.0017470755662254778, P82=0.0017411920925892667, P83=0.002518109790670702, P84=0.007862868941601127, P85=0.019547200103917684, P86=0.04410590802985001, P87=0.14559017334452073, P88=0.2755324588600225, P89=0.6437722596455857, P90=1.8704792175537872, P91=3.19690710346676, P92=0.921412527362982, P93=0.3244554065153835, P94=0.24969108352011948, P95=0.07878470943121788, P96=0.07888577680440076, P97=0.0972254099955273, P98=0.08580532407830797, P99=0.06213512140950281, P100=0.052079249196122945, P101=0.032151886674321825, P102=0.05949042453905646, P103=0.09375299208776625, P104=0.07169269847052469, P105=0.11297891115981556, P106=0.11694211782745706, P107=0.16898066771322398, P108=0.3696628281737255, P109=0.6499820961591014, P110=0.14732378841668087, P111=0.3043823619902705, P112=0.2819141809141924, P113=0.06553390990123374, P114=0.0770568475929296, P115=0.12458552257414553, P116=0.14995633865453054, P117=0.18714440623478942, P118=0.23460417128923372, P119=0.13003834049664345, P120=0.04247676905700563, P121=0.021200386314732613, P122=0.018973033651324054, P123=0.03389614174936149, P124=0.028875558164907604, P125=0.01841917842951493, P126=0.006403303391630228, P127=0.002736168705578183, P128=0.0013376682622319586, P129=0.0016745144268117183, P130=0.01114416590714234, P131=0.009667060326577366, P132=0.001340149195864316, P133=0.0016427135657230676, P134=0.002155260937797083, P135=0.0025335198588290267, P136=0.0024748129886741316, P137=0.002774791508866998, P138=0.003903532068733198, P139=0.008386610006027964, P140=0.0073134672546513345, P141=0.008253544254655423, P142=0.022727134135424872, P143=0.017055489769356582, P144=0.009838428644644487, P145=0.010442746811270553, P146=0.00905507191644336, P147=0.005509859180475332, P148=0.007304634538116974, P149=0.006955162335234635, P150=0.0025390545717426626, P151=0.0014169771073926625, P152=0.0005495344249105443, P153=0.00022309283270828705, P154=0.00035706125348921024, P155=0.0003726238004053063, P156=0.0003993318500313497, P157=0.00045930409076280564, P158=0.0004214197014396537, P159=0.0003994139420021579, P160=0.0004624042158643951, P161=0.000515281011061734, P162=0.0008365524816826392, P163=0.0007291187460007656, P164=0.00047143560437321004, P165=0.00031529902495886283, P166=0.0001864732322083786, P167=0.0002000991029801814, P168=0.00027105568957658916, P169=0.00031988987337308974, P170=0.0003084024172779123, P171=0.00015473869592011892, P172=0.00010570684720531004, P173=0.00012117736701631776, P174=0.0001100024815123182, P175=9.418120433742885e-05, P176=7.418858832081055e-05, P177=0.0001042080130575327, P178=7.976844027846035e-05, P179=9.236607218997505e-06),Row( P0=-645.6933428896996, P1=48.97871487874267, P2=-18.7065358808242, P3=-8.322907645682557, P4=-15.946249016292755, P5=-6.39134532885035, P6=-10.798175218592693, P7=-8.33007807050318, P8=-2.007107520586704, P9=-9.721729619271697, P10=-14.357047206435665, P11=-7.131395470219527, P12=-3.4172312746391555, P13=-6.869127322095347, P14=0.3515558770169357, P15=-0.5669651644718291, P16=-0.4329852228068116, P17=0.6791236488967819, P18=4.088588226811185, P19=4.209707893690718, P20=0.013359576074383655, P21=4.374514391833548, P22=1.2999998333814338, P23=-1.3698271523020897, P24=-1.107463921006413, P25=-3.324705917391011, P26=-2.3594316433810842, P27=1.0517055845615129, P28=-2.126495489708973, P29=0.0010703739632646225, P30=-2.7653114075523133, P31=0.3325654541353215, P32=-1.2710530331242067, P33=-0.21234937380430646, P34=-1.3169779724969752, P35=-0.3126357431315355, P36=-1.9517580833611679, P37=0.16570742902750732, P38=-1.5006036945168115, P39=0.5681728235451722, P40=0.6635993018682359, P41=0.6458566959454309, P42=0.6799954559830058, P43=0.6953909631618846, P44=0.6472742369057635, P45=0.6566782071927426, P46=0.6636683520289282, P47=0.6905840538519838, P48=0.7063365629651844, P49=0.7054150188039905, P50=0.6978864143005636, P51=0.6561431772447308, P52=1.391742021873747e-06, P53=3.479175013453713e-06, P54=1.3373095084085147e-05, P55=2.8782595481015392e-05, P56=5.327617022730176e-05, P57=0.00010046756778329212, P58=0.00011997030372762915, P59=0.0002653900063820885, P60=0.0006661600790641858, P61=0.0025002914003705076, P62=0.011625617154631125, P63=0.03334045250881808, P64=0.04092031493831104, P65=0.06704349324037519, P66=0.07029695372915083, P67=0.053498534184357, P68=0.029244045777180018, P69=0.0357156879129377, P70=0.028374186774620797, P71=0.007783066614088177, P72=0.0017381023293939648, P73=0.0012819339755734797, P74=0.003977308266109244, P75=0.012028755584603592, P76=0.02750732331868528, P77=0.03872738597724367, P78=0.034293473020914074, P79=0.03932821729682488, P80=0.04823193653846402, P81=0.01842396297488775, P82=0.009040199874108186, P83=0.007979000000672537, P84=0.007916489141862744, P85=0.0094020414375345, P86=0.010592850073567544, P87=0.03806516686053697, P88=0.09944979010087326, P89=0.06186409678325228, P90=0.025629031791739403, P91=0.030387786674141333, P92=0.041415837201096026, P93=0.011427764521668927, P94=0.006061402338450633, P95=0.0066685948610194475, P96=0.004389971729618589, P97=0.007542093156126554, P98=0.007374366141685303, P99=0.007990843166809964, P100=0.010314287993786628, P101=0.012695744589994387, P102=0.0032081836586602894, P103=0.003550360234772914, P104=0.007630011068289142, P105=0.015094046858662592, P106=0.013717437308880517, P107=0.02522826294458662, P108=0.03755775591709314, P109=0.04253339310705334, P110=0.03251350949116146, P111=0.019688764470482675, P112=0.008172475187702349, P113=0.006403954982836426, P114=0.004410494656418943, P115=0.0023440428980586996, P116=0.0012666102762334554, P117=0.0010704605200816577, P118=0.001400723145707205, P119=0.0012694525411797585, P120=0.00031449114125913505, P121=0.00027795694211895424, P122=0.00046723877944716255, P123=0.0004486925678255811, P124=0.0001300099656155193, P125=0.00017660198113212568, P126=0.00045563655831083695, P127=0.00011524446322246342, P128=3.9258017378168135e-05, P129=4.440069096559024e-05, P130=3.1231399953329054e-05, P131=3.7515511972683414e-05, P132=9.94903481219731e-05, P133=5.7757647450412676e-05, P134=0.0001241141857188501, P135=0.00019067326378949018, P136=0.00018893865393477578, P137=0.00028678779481440027, P138=0.00025227252352908293, P139=0.0002352946603211722, P140=0.00012675166615396945, P141=0.00014402098353008864, P142=0.000162181483193549, P143=0.00013332418426186937, P144=0.00012030583126377901, P145=0.00011907828453712768, P146=0.00017799270277057066, P147=0.0002226539902701938, P148=0.0001984198929126302, P149=0.00019948301235190406, P150=0.00016805518578290752, P151=8.307883377118859e-05, P152=4.423556585578108e-05, P153=2.8616232664455368e-05, P154=4.120215313879723e-05, P155=8.242763744310473e-05, P156=0.00012448403796036183, P157=0.00023964006254095614, P158=0.000225299342051801, P159=0.0002085107000806565, P160=0.00016505332212879282, P161=0.0001551941021675789, P162=0.0002458929440512379, P163=0.00026074237943428657, P164=0.0002496173054436689, P165=0.00012895494798828596, P166=0.0001162231612822856, P167=9.510600197635202e-05, P168=7.9571243128445e-05, P169=7.61094404081082e-05, P170=5.4279223898474346e-05, P171=4.722476893635364e-05, P172=3.360210533368671e-05, P173=3.184993087567213e-05, P174=4.45764609175189e-05, P175=5.664596391502977e-05, P176=8.900541992713661e-05, P177=7.170492463797342e-05, P178=3.088873582323875e-05, P179=2.376963160946329e-06),Row( P0=-721.9962680173767, P1=45.77378864024874, P2=-3.2808744668258467, P3=-5.227039178963676, P4=-10.601593795045073, P5=-11.871801138340892, P6=-8.635822047123229, P7=-9.102895284283427, P8=-2.4493772786505295, P9=-13.379154275974864, P10=-9.998056840651962, P11=-8.859622534865506, P12=-9.275134022234928, P13=-10.025875413635944, P14=-8.606986221594894, P15=-2.7085990864338907, P16=-6.0513364391952225, P17=1.8245525100465938, P18=6.165241793066975, P19=7.027086525618561, P20=8.827378510498582, P21=7.075176054246215, P22=5.952006367089966, P23=2.701932913978621, P24=0.09284810369535552, P25=1.8847342359841264, P26=1.296775975702251, P27=1.9319172203000083, P28=-1.8684020496213702, P29=-1.4588106304264377, P30=-4.217423187811234, P31=-1.0873701942105554, P32=-1.5983277294116813, P33=-0.5617878957493136, P34=-3.7644469301580603, P35=-2.67170007672758, P36=-4.593504620399576, P37=2.3479416398132695, P38=-0.7701413941628648, P39=0.6579552372384146, P40=0.5794557889671572, P41=0.5705757256232673, P42=0.5780937554665954, P43=0.6372038126172165, P44=0.689530543899707, P45=0.6798152068457102, P46=0.663801851262407, P47=0.6372482699640478, P48=0.6596122006111085, P49=0.7021757171787519, P50=0.665001377851369, P51=0.5924058492233081, P52=3.1251770167276976e-07, P53=8.455854058559946e-07, P54=3.410802449195416e-06, P55=7.415640509985616e-06, P56=1.1784735855033819e-05, P57=1.567310166510645e-05, P58=9.970653206019943e-06, P59=6.123087884985595e-05, P60=0.0022741582381131165, P61=0.01414941727891096, P62=0.03742645914074116, P63=0.042003229318388784, P64=0.04386450822092307, P65=0.08880710223165394, P66=0.05484889473780674, P67=0.019139616007328282, P68=0.0030315052306918527, P69=0.00032412167478721083, P70=0.0002083351647187464, P71=0.000437623265611292, P72=0.0008819207077257437, P73=0.001144614097046895, P74=0.0021567551636774117, P75=0.002719928244778929, P76=0.002991664064368145, P77=0.009793566493340665, P78=0.01165861825755889, P79=0.008438344912804886, P80=0.006418366450380572, P81=0.004540971765750907, P82=0.003695059958664201, P83=0.001290826573242028, P84=0.0012843227842922334, P85=0.0016901396849617878, P86=0.0009635529075018611, P87=0.0006327339191887782, P88=0.0009030267122651069, P89=0.0026525067588029447, P90=0.003928844094215362, P91=0.0035887131016545657, P92=0.0013093086049760431, P93=0.0005539724844256939, P94=0.0005580841676828649, P95=0.0003351123197763085, P96=9.873336454952592e-05, P97=0.00025880533726278066, P98=0.00034138161368916186, P99=0.001600215231457801, P100=0.002775035713819661, P101=0.0011823417656705779, P102=3.90898677005955e-05, P103=2.2003400226428238e-05, P104=3.076215578067136e-05, P105=8.569684565219045e-05, P106=0.00023383769504167854, P107=0.0002293992551680132, P108=7.665999970760457e-05, P109=3.4980865443680615e-05, P110=4.007359910363842e-05, P111=5.671118300140983e-05, P112=0.00020255551039460534, P113=9.69625929732568e-05, P114=2.4485078013365533e-05, P115=3.658586278691317e-05, P116=0.00011026027389346424, P117=0.00034938305672287654, P118=0.0003971204626171346, P119=5.719121075864774e-05, P120=9.138821404706418e-05, P121=0.00015529952383905998, P122=0.00018884522796445437, P123=3.563502425239536e-05, P124=3.890742502021304e-05, P125=2.3925088615199548e-05, P126=9.639569286053193e-06, P127=1.370927307701184e-05, P128=6.098851962446968e-06, P129=5.412184267822008e-06, P130=5.470876636133437e-06, P131=7.269161462963706e-06, P132=3.0095132278925482e-06, P133=3.4702682883712505e-06, P134=4.0532688209575265e-06, P135=3.6833001020526483e-06, P136=4.899479802567192e-06, P137=5.5557099292858405e-06, P138=5.710783070183901e-06, P139=7.710050149060806e-06, P140=1.2209880430789376e-05, P141=1.3246395820695473e-05, P142=9.677645371543117e-06, P143=1.2279778536760712e-05, P144=6.396388309253304e-06, P145=3.979905728968209e-06, P146=2.624863638560908e-06, P147=4.0674334312172835e-06, P148=4.696264758224022e-06, P149=4.024859676164442e-06, P150=4.9335665106068605e-06, P151=8.10133774976753e-06, P152=4.899167102874156e-06, P153=4.051735436863604e-06, P154=7.416939246242134e-06, P155=1.0966483942354604e-05, P156=1.0974735496601601e-05, P157=1.1854182584200577e-05, P158=8.863483953352788e-06, P159=1.1276229024586043e-05, P160=1.145409672788192e-05, P161=1.3595897489256178e-05, P162=1.9070280433298804e-05, P163=1.6436135250774918e-05, P164=1.5663801255115245e-05, P165=1.4412650133250792e-05, P166=1.1133879926359009e-05, P167=1.2764186029563395e-05, P168=2.2049232827867522e-05, P169=1.3934167619948065e-05, P170=9.095462284999587e-06, P171=8.62722130989107e-06, P172=8.898398836461941e-06, P173=1.3966327013513829e-05, P174=1.4805084315224112e-05, P175=1.9585989689771762e-05, P176=1.4292132406004305e-05, P177=1.3356290139633766e-05, P178=7.1954164888320755e-06, P179=6.76988396318083e-07),Row( P0=-733.8516660186503, P1=42.981620737134705, P2=-7.062614160874111, P3=-2.312533677889069, P4=-15.746778383837867, P5=-11.115777134871978, P6=-8.339596115366101, P7=-11.409963187481514, P8=-0.980870712994463, P9=-11.635074313298086, P10=-9.311399645530477, P11=-7.57461895645124, P12=-8.896156812623849, P13=-8.10021128612915, P14=-6.49233356537412, P15=-1.1021778686730626, P16=-3.4238347629061727, P17=4.813592547687456, P18=7.541017274895025, P19=9.19136184708908, P20=8.784455308431411, P21=6.876855085090613, P22=4.0970158929722436, P23=2.5748293528219186, P24=-0.416414044900455, P25=0.5592553416651843, P26=0.4505580127669603, P27=-0.38874523299454666, P28=-3.875016858878818, P29=-2.660574009947442, P30=-3.768237727829435, P31=-0.19986785271374793, P32=-2.0048984320352394, P33=-0.561201372824151, P34=-3.8290904129591534, P35=-1.0808661104083772, P36=-2.5375631477228078, P37=2.8219490389611037, P38=-0.8586343081756441, P39=0.2314816643286865, P40=0.5812108349736526, P41=0.5614828424198459, P42=0.5743082516970203, P43=0.5980285468020374, P44=0.6496859746038082, P45=0.6597464024565719, P46=0.6395481386078149, P47=0.6559723691696687, P48=0.690210646994157, P49=0.7362584456170305, P50=0.720620474871048, P51=0.6032131197233791, P52=5.892454158444336e-07, P53=1.6568539128050441e-06, P54=2.599824442795529e-06, P55=6.934326053968207e-06, P56=1.5489272693905275e-05, P57=1.0527357383360897e-05, P58=2.7231812967875856e-06, P59=3.877231293490664e-06, P60=0.00017014326873719435, P61=0.003228966768742286, P62=0.013217352606946755, P63=0.022437918655830644, P64=0.029487427856429375, P65=0.10081792460824701, P66=0.07169087301689277, P67=0.006808231460814431, P68=0.0007930555611171456, P69=0.0002408792789518338, P70=0.00010849326897788552, P71=0.00010687137034978182, P72=0.0005000739397082801, P73=0.0005824337258917558, P74=0.0013445887716799154, P75=0.0026918187283046804, P76=0.004625284993482744, P77=0.00782807624293908, P78=0.0045210806271672, P79=0.008929746026720288, P80=0.01748680907827294, P81=0.0074248309744063806, P82=0.001466611267433959, P83=0.0005082598422623151, P84=0.00048295062606523763, P85=0.0005959226907743624, P86=0.0006466157654996699, P87=0.0008742707293437978, P88=0.0016232752185577363, P89=0.0024797812268710392, P90=0.0011696542147738413, P91=0.0015323752566508253, P92=0.0015488042512014815, P93=0.000549200185674085, P94=0.00016138211189670046, P95=0.00012880314737628084, P96=0.00017377084577706406, P97=0.0007476079385546448, P98=0.0004364561489492386, P99=8.475502646089658e-05, P100=0.00028504060073244515, P101=0.0012026429565412003, P102=0.00025445465985982947, P103=4.467156775091267e-05, P104=8.948570239461426e-05, P105=8.239457897238825e-05, P106=2.6170097072755977e-05, P107=0.00033034698699030407, P108=0.0010941489965645452, P109=0.00027296827666097094, P110=0.0001236264843933479, P111=3.513275871722873e-05, P112=4.848530642545566e-05, P113=0.00017727355046219003, P114=0.0001667342487279335, P115=6.64945870017821e-05, P116=3.632120656884122e-05, P117=4.5455718703511395e-05, P118=0.00047321337230171555, P119=0.0003064282382638097, P120=4.0701976933788866e-05, P121=3.446638695598242e-05, P122=4.236855249344682e-05, P123=4.262165475659557e-05, P124=1.7980870995589557e-05, P125=1.5323460648576442e-05, P126=1.3892828005854782e-05, P127=1.1801120243317974e-05, P128=1.3781187306251558e-05, P129=5.894215385303314e-06, P130=4.552175838864494e-06, P131=3.4027387669297646e-06, P132=3.6808257840230176e-06, P133=2.4091640202774614e-06, P134=3.156819317329685e-06, P135=5.528991963571409e-06, P136=4.392820548724659e-06, P137=8.63972919510378e-06, P138=9.103796761469451e-06, P139=2.2520010670725325e-05, P140=1.852229374732155e-05, P141=2.9140762291260207e-05, P142=2.2447603099274443e-05, P143=1.173835792625702e-05, P144=1.2386318147638838e-05, P145=6.826626173740223e-06, P146=6.478459240025325e-06, P147=7.85403515465241e-06, P148=1.133215368017668e-05, P149=1.8418378555250002e-05, P150=1.7796181186405635e-05, P151=2.864246415853877e-05, P152=2.0754920680123138e-05, P153=1.1776582152585798e-05, P154=1.1875302870562726e-05, P155=2.0480102977727484e-05, P156=2.502084792302675e-05, P157=1.3488768915435977e-05, P158=1.0190844673306076e-05, P159=9.934490597116009e-06, P160=7.897364394663057e-06, P161=6.443925443626422e-06, P162=3.789001901509846e-06, P163=3.2582573819121656e-06, P164=2.6076327559986588e-06, P165=2.213704139773e-06, P166=1.3602121226278872e-06, P167=1.7412022626874717e-06, P168=3.27357024259438e-06, P169=2.911221806456075e-06, P170=1.743691578190796e-06, P171=1.2825251745997724e-06, P172=1.3678764360958483e-06, P173=2.8125252987904946e-06, P174=4.366800237259468e-06, P175=3.929270368331062e-06, P176=2.8249806502958266e-06, P177=3.229239604973206e-06, P178=1.3753481803740394e-06, P179=1.294854833816922e-07),Row( P0=-652.3874072654205, P1=34.91790176635911, P2=-24.82759619771943, P3=-3.9986239816422273, P4=-18.23862450793784, P5=-16.707778565386757, P6=-7.333954991955455, P7=-6.974824496199294, P8=-3.173327422651288, P9=-14.97618374605437, P10=-8.50317287009742, P11=-1.273112847687728, P12=-1.9705509540163844, P13=2.8524089105668207, P14=10.199499785237283, P15=12.358130168363093, P16=2.6716633460695935, P17=2.784124409537238, P18=0.29010672395560977, P19=-3.2384539702638686, P20=0.4280602154142757, P21=2.198118637238273, P22=-0.4482357736809126, P23=-2.368214096064977, P24=-6.344105937378248, P25=-3.9538582440555956, P26=1.849658340672364, P27=3.119525351553435, P28=-1.9517974605743853, P29=-0.48378998726176287, P30=-2.493519611319717, P31=0.0903176359711508, P32=-1.4900282238317328, P33=-1.7305081263056377, P34=-2.0461560724238015, P35=-2.036064347211731, P36=-1.8015997154029175, P37=0.48885748893655506, P38=-1.0588447670957635, P39=1.8384833363900768, P40=0.6423875810615401, P41=0.7033328751180996, P42=0.7239350231885804, P43=0.7170086339890698, P44=0.6790243812426315, P45=0.5901564119693231, P46=0.5506051036622802, P47=0.5719730899084058, P48=0.6505258302835394, P49=0.7057486295635352, P50=0.6771894316832965, P51=0.6029174048434751, P52=1.34088054636456e-06, P53=4.465808668037702e-06, P54=1.3452337101478899e-05, P55=1.9001886332973947e-05, P56=4.064010622582117e-05, P57=5.8662259655696767e-05, P58=3.890853730553213e-05, P59=2.6090632057179e-05, P60=2.6535726780110973e-05, P61=3.2551185125930746e-05, P62=0.00021524054111809644, P63=0.0014634504855752415, P64=0.009912713889136504, P65=0.023504721234306854, P66=0.02086767109907384, P67=0.025571251853798486, P68=0.055898027670122113, P69=0.18019502757251313, P70=0.1646443793849448, P71=0.08201382771019204, P72=0.040083060092217085, P73=0.011950315853657991, P74=0.0022209160688129977, P75=0.0010384112553920375, P76=0.0008466821910460273, P77=0.004153543796334632, P78=0.010894285478641644, P79=0.009309801859922383, P80=0.0031360983632166377, P81=0.0014785715210693052, P82=0.0022107135801694446, P83=0.011746144318069177, P84=0.058209051169943925, P85=0.09357410955239223, P86=0.03417375898896048, P87=0.04752598639595777, P88=0.0714526800017756, P89=0.0625074922595935, P90=0.06779440386330052, P91=0.03773270550840991, P92=0.007891812441572395, P93=0.0019636501818488363, P94=0.0027839530128779856, P95=0.002912568784816095, P96=0.001767198108309012, P97=0.0048464361361167515, P98=0.010435815050576383, P99=0.010647545664297403, P100=0.01748407736775625, P101=0.016991680784720624, P102=0.004471502372981134, P103=0.0011835007932290136, P104=0.0004108448270821777, P105=0.0004752503673217809, P106=0.000717242010512487, P107=0.0042889060393149925, P108=0.0037098075274143083, P109=0.00846417502699082, P110=0.005530047687080979, P111=0.0017882066644736784, P112=0.001423853439254097, P113=0.0005475641822545678, P114=0.0023534384419055367, P115=0.004512913072809423, P116=0.003619957118098788, P117=0.0020715958684034, P118=0.0012955351995688473, P119=0.006691033079854277, P120=0.009246620088121759, P121=0.007822879208593767, P122=0.0054788997200074475, P123=0.0022213103212752293, P124=0.0018117998004698907, P125=0.0010074997688879314, P126=0.0008334635777773931, P127=0.0012530781402273125, P128=0.0003448890450426142, P129=0.0002116226996574834, P130=0.00014522899692222385, P131=9.607847630018961e-05, P132=6.716123171710209e-05, P133=7.822473597615098e-05, P134=7.011922928217752e-05, P135=9.250340879788891e-05, P136=0.00010547970629786165, P137=0.00016611961320180455, P138=0.00019600662152496597, P139=0.0002857880629768844, P140=0.00026883703140745044, P141=0.0001863458752998072, P142=0.00018598293715089907, P143=8.289314330616795e-05, P144=3.9709780719371085e-05, P145=3.266999944333851e-05, P146=2.7860854321040314e-05, P147=4.232030623674777e-05, P148=5.0428419719724165e-05, P149=6.913303357177191e-05, P150=7.844059852082317e-05, P151=6.072354568430601e-05, P152=4.169040868146071e-05, P153=1.7006593317000696e-05, P154=1.3570908417417595e-05, P155=3.173927015354333e-05, P156=5.261137954350444e-05, P157=5.816191661482392e-05, P158=9.266162010172579e-05, P159=7.609379278872985e-05, P160=4.8711495730585794e-05, P161=3.892488019186319e-05, P162=2.600933482757834e-05, P163=3.916768338252788e-05, P164=2.5428783187778835e-05, P165=3.639981667047025e-05, P166=3.473895934362992e-05, P167=1.2550222218656455e-05, P168=8.581210524281969e-06, P169=8.38680326146933e-06, P170=8.307919024606101e-06, P171=1.0795668346478029e-05, P172=1.0841500609725348e-05, P173=9.079828133459633e-06, P174=9.313125877330597e-06, P175=8.945992599953449e-06, P176=1.1770020387139528e-05, P177=1.8002739643161644e-05, P178=1.2301609173011312e-05, P179=1.4046689254223343e-06),Row( P0=-675.2201165107961, P1=38.35791627193605, P2=-17.58562243080055, P3=-8.421019522069853, P4=-19.323016428620722, P5=-17.229586110428894, P6=-8.88131215929814, P7=-11.46400984589872, P8=-6.119491511726825, P9=-14.669012965025441, P10=-10.49206342868114, P11=-5.504073855605106, P12=-4.237276149809433, P13=0.9048128002841385, P14=6.4776095983391375, P15=10.006904684002654, P16=2.7992953501452424, P17=3.0834397748724283, P18=1.2985101398214356, P19=-2.2824257741975056, P20=-1.470760579550218, P21=0.34150303298367024, P22=0.23342021143277358, P23=0.1421993059875871, P24=0.5559183143958354, P25=0.6823664667449643, P26=-0.8501957152282625, P27=-1.6283806498539926, P28=-2.8403837745148013, P29=0.9577399631202317, P30=-1.5169975909109368, P31=0.7474780416258717, P32=-2.0337718963156792, P33=-0.721901975251955, P34=-0.7456585030985002, P35=-1.0129012194582225, P36=-4.222617026134042, P37=-0.9730303988910742, P38=-1.396281150283709, P39=1.427375085641598, P40=0.6868351813946261, P41=0.6668912988661418, P42=0.6561141832087952, P43=0.6511451703139697, P44=0.6499614288358925, P45=0.6487629757366126, P46=0.6184126646393349, P47=0.6009768322668334, P48=0.5571048212423189, P49=0.5562652467760733, P50=0.5817694764082711, P51=0.6327189961913299, P52=2.9546013902337774e-07, P53=7.687243098139707e-07, P54=4.699498639055712e-06, P55=1.3101448309666698e-05, P56=2.5765152130453212e-05, P57=2.7391308687227315e-05, P58=1.1198957661896908e-05, P59=5.065794094618242e-06, P60=6.1485057113264e-05, P61=0.0015046752832490009, P62=0.008256866081044879, P63=0.010834657198789722, P64=0.00497965331982066, P65=0.014241319026039222, P66=0.05494407490872422, P67=0.10797893130237741, P68=0.06910628720999656, P69=0.07124088885341412, P70=0.062433183283201556, P71=0.0414904901347866, P72=0.05307582278865816, P73=0.03420769361112532, P74=0.00990854207158722, P75=0.004835706364768282, P76=0.0023164429308681913, P77=0.002028581902864485, P78=0.00225944938470832, P79=0.002194635842500084, P80=0.0027652762090325123, P81=0.003534863955852304, P82=0.012950051357148681, P83=0.03711780307041706, P84=0.06192549172585514, P85=0.05964918454272633, P86=0.0169360138008593, P87=0.012940930060732039, P88=0.025876892739287814, P89=0.04986308680588015, P90=0.03244896084429035, P91=0.03841679776845068, P92=0.021134145478995336, P93=0.0046953246937157635, P94=0.011398500643284881, P95=0.02336971582391854, P96=0.007781793305785018, P97=0.0050261447927611, P98=0.00456089029523285, P99=0.002744610737720063, P100=0.0031511601631729193, P101=0.0019551301973452467, P102=0.0010840672968660415, P103=0.0015762110801204564, P104=0.0025989589150798025, P105=0.0017334238682008905, P106=0.0010392894080924323, P107=0.0002542342614376038, P108=0.00019924638215614121, P109=0.0003986609619481571, P110=0.00039281483669553654, P111=0.00044076817595274697, P112=0.0005897159857268944, P113=0.0004513076438171825, P114=0.00020360688203400725, P115=0.00047280252144556563, P116=0.0005152335523180805, P117=0.00043135197022238216, P118=0.00040195095150606446, P119=0.0004793616234531884, P120=0.0013250081689168208, P121=0.0017959243948740748, P122=0.0010624430948161505, P123=0.0012811212414511781, P124=0.0012476516665804952, P125=0.0018856882670809797, P126=0.0012949733043364127, P127=0.00048386759469528826, P128=0.00023008478363993506, P129=0.00011205110146300773, P130=0.000196052658046046, P131=0.00013963857900634223, P132=0.0001934484177284572, P133=0.0001342564618311461, P134=0.00012989630651787844, P135=0.00015868566000471016, P136=6.98325151141284e-05, P137=6.835287422490452e-05, P138=0.00010706098091076124, P139=0.0001237258053086287, P140=0.00013373422042792804, P141=0.00012426626522515295, P142=0.00011303922763078942, P143=5.078098899816802e-05, P144=3.770957991461037e-05, P145=1.7723431212981964e-05, P146=2.318767972667965e-05, P147=3.287136976976831e-05, P148=4.47711428970189e-05, P149=3.8204717463725944e-05, P150=2.8598725059124942e-05, P151=2.3685100438284955e-05, P152=2.3237841246521775e-05, P153=1.6199642052939195e-05, P154=1.06654621955292e-05, P155=7.752996519245995e-06, P156=1.8399162863049908e-05, P157=2.615247473812039e-05, P158=3.0061059082866748e-05, P159=5.709636906761327e-05, P160=5.225493367277353e-05, P161=4.55134303907818e-05, P162=4.012586374676926e-05, P163=0.00010465762917076655, P164=6.659412182188563e-05, P165=5.2003716496165495e-05, P166=5.355581533726619e-05, P167=8.410619216515751e-05, P168=9.490422006559855e-05, P169=6.790880384160961e-05, P170=8.862668200007264e-05, P171=4.413146660382944e-05, P172=4.572794592237787e-05, P173=8.673755098889892e-05, P174=5.5095091300574685e-05, P175=7.887125263477918e-05, P176=8.415849901441749e-05, P177=6.496689381132323e-05, P178=2.533256528254e-05, P179=2.903315739935471e-06),Row( P0=-710.4601979297481, P1=42.634641320298456, P2=-5.947014068128131, P3=-7.5922263611195415, P4=-9.941372511132553, P5=-12.740239172249641, P6=-8.755205250336699, P7=-9.506713504622873, P8=-0.8503861363519862, P9=-11.603557238258299, P10=-8.123414355775527, P11=-8.96997391616674, P12=-9.944711838177232, P13=-8.769614299122827, P14=-6.687034514477222, P15=-1.049870670341722, P16=-4.180296983913583, P17=1.4496233744969764, P18=5.2191758721211725, P19=6.640464749865755, P20=6.478408776401422, P21=5.924885749871375, P22=4.611233907005973, P23=4.270226633919208, P24=2.118003708030635, P25=2.1733271756636587, P26=1.8077726410284594, P27=0.7065562942710729, P28=-3.802300651310616, P29=-1.4273591353637953, P30=-2.2192181284231975, P31=-0.4301842632659722, P32=-1.5876020181431827, P33=-1.561566994512634, P34=-3.886106581021311, P35=-0.3311328994976184, P36=-2.3623752977143475, P37=1.9257661130693822, P38=-2.5040534885985832, P39=-1.6265069104186232, P40=0.5981521940851403, P41=0.6185296885489083, P42=0.616728514332356, P43=0.6272933474032979, P44=0.6876230213844636, P45=0.667608691489568, P46=0.6392556575228835, P47=0.6205403009673761, P48=0.628291533484954, P49=0.6621517326156371, P50=0.6578035680242058, P51=0.6093072981562808, P52=5.178711367425677e-07, P53=1.9222059217311883e-06, P54=6.8781714529802634e-06, P55=4.888497588457429e-06, P56=7.345374976637717e-06, P57=8.238614768411448e-06, P58=5.6426501837174644e-06, P59=2.3031179714950614e-05, P60=0.0015280811897478994, P61=0.011774347483485053, P62=0.040142112311985785, P63=0.06200778973561744, P64=0.03810162452391971, P65=0.08116905849059002, P66=0.07072628873397288, P67=0.018395082787343642, P68=0.0020575126371299387, P69=0.00044988638958378947, P70=0.00024210420039032018, P71=0.0006485726514738336, P72=0.0011033917863783342, P73=0.0017392944405756133, P74=0.007898903911571878, P75=0.012598232827900006, P76=0.008621488875841504, P77=0.00613289408786892, P78=0.012130981679740062, P79=0.011921344715383123, P80=0.013113803222863712, P81=0.012581727922380504, P82=0.00703828800901866, P83=0.002037276606570401, P84=0.002014992689491867, P85=0.004345631597165466, P86=0.005164218860752369, P87=0.00881301485326031, P88=0.0043584266594103626, P89=0.0008585250492156598, P90=0.0013490925938236724, P91=0.0025229267030178024, P92=0.0032324266723314497, P93=0.0016964183994842934, P94=0.0018577217913472949, P95=0.000719745226153913, P96=0.00015623813822044136, P97=0.0003199443053839306, P98=0.00026470307145461645, P99=0.0003287627640969293, P100=0.0019206137659613476, P101=0.00277942098835428, P102=0.0006174536056347197, P103=0.0001902633636237581, P104=8.870637166020186e-05, P105=0.00012848499070005584, P106=0.00015556856208419966, P107=0.0011866791123138572, P108=0.002519793710701583, P109=0.000218625832840941, P110=3.5243619685302375e-05, P111=0.00010057246714863931, P112=0.00022503145913740994, P113=0.0003559010490646768, P114=0.00021622253762942204, P115=0.00017164735084699081, P116=8.630042338785862e-05, P117=0.00018776503974037023, P118=0.0012096653971431265, P119=0.000865101262305708, P120=0.00023073361660396212, P121=0.0002296375427074475, P122=0.0006073864986383951, P123=8.631460606688982e-05, P124=8.087659498859944e-05, P125=5.700441999189433e-05, P126=2.386278482983982e-05, P127=1.8891849130516676e-05, P128=2.131305226082486e-05, P129=9.029551017464695e-06, P130=1.0922579173071496e-05, P131=1.1992141641693167e-05, P132=6.011313246681617e-06, P133=3.928908648649668e-06, P134=5.2345840830416425e-06, P135=6.6144024235857425e-06, P136=6.311670881852639e-06, P137=1.2542074732201443e-05, P138=1.5767841927572408e-05, P139=2.5154350410339183e-05, P140=2.1820939208169182e-05, P141=1.4324013438863163e-05, P142=8.756834747024601e-06, P143=6.4001095926308574e-06, P144=3.3062742816703256e-06, P145=4.8266792152438315e-06, P146=5.443190527312206e-06, P147=8.184295257744964e-06, P148=9.421089114456008e-06, P149=9.345874708298663e-06, P150=7.72958006666667e-06, P151=6.430102367573408e-06, P152=4.95601644392532e-06, P153=6.1419094869179825e-06, P154=7.218601739755969e-06, P155=1.0073687730132742e-05, P156=9.364410904930545e-06, P157=1.0891884807276629e-05, P158=8.36056754988238e-06, P159=7.76282668606359e-06, P160=8.759289994885309e-06, P161=1.2137786700879399e-05, P162=1.580005338942001e-05, P163=1.5232339760667721e-05, P164=1.622920536603799e-05, P165=2.480904152837458e-05, P166=1.5007227911775497e-05, P167=2.2216154362455657e-05, P168=3.84420406133558e-05, P169=2.5851064644495964e-05, P170=1.8944399207423287e-05, P171=2.6474394077860487e-05, P172=2.0359876363008108e-05, P173=1.9648040132068613e-05, P174=3.667380316504032e-05, P175=3.7135218244268824e-05, P176=3.365872546663405e-05, P177=2.6279915276059344e-05, P178=1.019788981999505e-05, P179=1.1435435116936917e-06),Row( P0=-594.720655786716, P1=38.25128082347314, P2=-22.61962430353602, P3=-14.495835202271309, P4=-12.830237845202795, P5=-14.426660130794463, P6=-10.47903951567915, P7=-5.96184039902239, P8=-2.361587784046685, P9=-10.132953279086767, P10=-12.340120610456614, P11=-3.909520841861553, P12=-3.1173130008091574, P13=-2.7301220084107873, P14=5.4066568380078825, P15=4.346058505576416, P16=6.485588358503287, P17=4.350405254014708, P18=4.27976868229479, P19=3.377875160355148, P20=0.2174900514275801, P21=2.2805182580912, P22=0.14982254280554327, P23=0.6839563431956356, P24=0.032179488171769126, P25=-1.2375203711213993, P26=-1.6985839814483947, P27=0.11047860929747493, P28=-2.477642980400076, P29=0.8873443790493988, P30=-0.6226067480007588, P31=1.5912983900114956, P32=-0.9731016320781603, P33=-0.9167764779355316, P34=-1.9493819390510194, P35=0.5602542713129215, P36=-2.679288443288247, P37=-1.27051194785524, P38=-1.9681361724067346, P39=-0.1173812180430131, P40=0.6702947061235349, P41=0.6482468306081608, P42=0.6311112206389609, P43=0.6221219403468803, P44=0.6293198350754582, P45=0.6080409500908042, P46=0.6091536429025176, P47=0.6068224095381274, P48=0.6207748542637077, P49=0.6027602296118002, P50=0.5955703181232148, P51=0.6394568491018408, P52=3.4594379505678525e-05, P53=2.887669558209313e-05, P54=3.146273078057195e-05, P55=4.2236001141725156e-05, P56=4.552281037293879e-05, P57=4.559545478425261e-05, P58=3.008759053095748e-05, P59=4.211010808155961e-05, P60=0.0005369678429177979, P61=0.003211675166323614, P62=0.013049313107980605, P63=0.01967402650711614, P64=0.021652368807498823, P65=0.06884448823021327, P66=0.14151387389244205, P67=0.1573191233477494, P68=0.11416598798397508, P69=0.1099689611701228, P70=0.07234090513668215, P71=0.06256926621173212, P72=0.033433665666610816, P73=0.012482358635257521, P74=0.017139967598464, P75=0.014036759628144517, P76=0.007555674864115183, P77=0.01837491766462855, P78=0.04555471816723042, P79=0.04635187590135917, P80=0.050076922490997676, P81=0.06448622636712444, P82=0.0728775411229643, P83=0.24757740712169543, P84=0.3469602430276358, P85=0.3724258490366253, P86=0.3096948561619162, P87=0.14387005213910306, P88=0.12052046397117049, P89=0.0942489813143468, P90=0.13707537473764658, P91=0.08199254431407844, P92=0.048607293397881696, P93=0.08094805159498514, P94=0.12982085611382854, P95=0.08125043125226834, P96=0.02657094822056694, P97=0.02589693024418435, P98=0.022722078543645566, P99=0.013066284019642634, P100=0.011623245441698456, P101=0.009086881249994565, P102=0.010713904986153001, P103=0.006537522426083738, P104=0.014346004612637147, P105=0.028373386592401323, P106=0.01618572895520054, P107=0.006271017630636934, P108=0.0058864234042095885, P109=0.018493383759615997, P110=0.016649403077242768, P111=0.010914683434480864, P112=0.007371654071171035, P113=0.006916317351618038, P114=0.011356054243353463, P115=0.02202933894869012, P116=0.017746006110017143, P117=0.006579441740459246, P118=0.0034674363439308902, P119=0.007957262990038069, P120=0.005239394407454732, P121=0.002474969230511402, P122=0.004064947715743055, P123=0.010057208856442252, P124=0.0074835167770956265, P125=0.002205740842097734, P126=0.002393248726295134, P127=0.001850370933265919, P128=0.0008889511373936146, P129=0.0005376540731545737, P130=0.0007160530347427747, P131=0.0004106728065214034, P132=0.00020672527003859578, P133=0.0004092089086964359, P134=0.00039125400011768245, P135=0.0002879380723802972, P136=0.0006047475421367902, P137=0.00029507732951364026, P138=0.0004785156510944067, P139=0.0003276064928388748, P140=0.00041663097630863027, P141=0.00018925128750641858, P142=9.502464985087406e-05, P143=0.0001226891107992463, P144=8.842524642751771e-05, P145=0.00012511097070009748, P146=0.000131525222417609, P147=0.00015485260987256453, P148=0.00015673852998580466, P149=8.070320208013823e-05, P150=5.256240281583322e-05, P151=5.3914276610700734e-05, P152=2.8119108655426656e-05, P153=2.9939450714852833e-05, P154=3.8681325742888695e-05, P155=4.25389005428304e-05, P156=4.9463939641745655e-05, P157=6.537153107670992e-05, P158=9.397019207414761e-05, P159=7.5183835299362e-05, P160=0.00011097104171589244, P161=0.00013112344605087367, P162=0.00014541007738940514, P163=0.0001541989705448175, P164=0.0001467640214595769, P165=0.00018151262425183032, P166=0.00018954986366917858, P167=0.00016621725841858088, P168=0.00014954350693659668, P169=0.0001441082560779215, P170=0.00021559077920636365, P171=0.0003527207896120998, P172=0.000442816078113213, P173=0.0006327726564998954, P174=0.0004766002637611225, P175=0.0003864656949782936, P176=0.0003689261151265449, P177=0.0005188470201494125, P178=0.00041849665356997085, P179=6.47755702924838e-05),Row( P0=-584.009460387514, P1=39.230104918027536, P2=-26.02562999735343, P3=-9.33889674927359, P4=-16.26975784051423, P5=-8.96847652781889, P6=-11.655110501610466, P7=-11.221415861907234, P8=-3.943442029272545, P9=-12.08261425188731, P10=-17.03119642069645, P11=-4.5041576365427245, P12=-5.929236629677954, P13=-5.961054120970001, P14=4.091272049168847, P15=0.11066555597071086, P16=1.9642210598753669, P17=4.025050860825247, P18=4.585808486524296, P19=3.570789544703985, P20=0.9400474690226861, P21=3.3543909315361717, P22=-1.396614066758154, P23=-3.3488139506288346, P24=-1.0088243522509743, P25=-3.775902204824563, P26=-0.34445644114666557, P27=0.6916707127789571, P28=-3.588101910069486, P29=-1.4250427507105567, P30=-2.80759408513669, P31=1.4841257000023, P32=0.11625220282900632, P33=0.09878196878796487, P34=-2.216716073589124, P35=-0.32648300742074243, P36=-0.29747475541842516, P37=1.7850791932049281, P38=-0.5844629103527137, P39=0.43920159022089766, P40=0.6463122074561657, P41=0.6477532126702141, P42=0.6539014169901448, P43=0.6690884670509427, P44=0.669776115368553, P45=0.6844992433932677, P46=0.6589651694434356, P47=0.6607243099520667, P48=0.6766663862134842, P49=0.6909809803756575, P50=0.725385021188911, P51=0.6838551690227193, P52=6.241366041742925e-07, P53=1.7646347064326214e-06, P54=9.255463065861306e-06, P55=2.2650269811935603e-05, P56=3.331621138540596e-05, P57=4.581853822690164e-05, P58=4.568057821931239e-05, P59=0.00027098655186831165, P60=0.0012140922101806912, P61=0.0032026257895237313, P62=0.0078296883408534, P63=0.016748452963687457, P64=0.03690686945986735, P65=0.13488960555002638, P66=0.14741519529944522, P67=0.07661389722618171, P68=0.03402297665710383, P69=0.04803983870802848, P70=0.0553265793631914, P71=0.03879525687920692, P72=0.029708030576479468, P73=0.01479517184002551, P74=0.004306121217292459, P75=0.003060070201123458, P76=0.004515131841541059, P77=0.01697965869181937, P78=0.04574868917477557, P79=0.13475388901366286, P80=0.13262160653415914, P81=0.09300535488597021, P82=0.09500284021953992, P83=0.04517639704675362, P84=0.019151309625910704, P85=0.01882065485562855, P86=0.01198453977108146, P87=0.02754516229281755, P88=0.057142620850082565, P89=0.07611405287052825, P90=0.08175520458800949, P91=0.12346135342091796, P92=0.09715309503113971, P93=0.04863100870122872, P94=0.0630528607446481, P95=0.0202552852824359, P96=0.009173962842565555, P97=0.03951489667925931, P98=0.045729943017301995, P99=0.031043597391268628, P100=0.05665259110498833, P101=0.026237807120312498, P102=0.010026811902303782, P103=0.005220547303196513, P104=0.0016907733021971347, P105=0.04009972471779979, P106=0.0716443370273418, P107=0.054761326612444365, P108=0.07527374112293812, P109=0.3507306868515922, P110=0.12789800812420998, P111=0.014592764266895432, P112=0.007862779577896968, P113=0.00923008446908758, P114=0.017691799371773797, P115=0.04207433744333514, P116=0.033867628290942, P117=0.005834770053394728, P118=0.0014877234313943805, P119=0.0017981156293760151, P120=0.0029575078904000365, P121=0.0018086903752894804, P122=0.001777777719152965, P123=0.0012423885629335224, P124=0.0023775897477393486, P125=0.0007093136472234566, P126=0.0003941077245311881, P127=0.0006729884043539349, P128=0.0009728774472384957, P129=0.00021542778840126462, P130=6.455027856596848e-05, P131=9.124904088697932e-05, P132=0.00011083528195601011, P133=6.732060347502155e-05, P134=0.0001877616148884562, P135=0.00011156516474090757, P136=0.000310961603284569, P137=0.00044615555599410254, P138=0.0003802273461711587, P139=0.0007981695254817084, P140=0.0005506342421241452, P141=0.0004293888876746949, P142=0.00031684717536901655, P143=0.0003267809113393769, P144=0.0003008557814304561, P145=0.0002674495728534695, P146=0.00018714230418459023, P147=0.00024389707104256118, P148=0.000274716196671107, P149=0.00023064944944416858, P150=0.00016963792974809572, P151=0.00011225281983258005, P152=6.843161754992214e-05, P153=3.6788206559144186e-05, P154=9.787076359321216e-05, P155=0.00013297439452965732, P156=0.0001461482139270136, P157=0.00023581995996714496, P158=0.00011532541780932435, P159=0.00011529249799867391, P160=0.00015419409728770997, P161=0.00013830115480300658, P162=0.00017463977900045497, P163=0.00013970848712318364, P164=6.591979222213637e-05, P165=8.302611424670589e-05, P166=9.105439544341789e-05, P167=5.4953006675092086e-05, P168=4.117354686453086e-05, P169=4.477636568111095e-05, P170=4.100008724062164e-05, P171=3.3199445071461726e-05, P172=2.6727348230141253e-05, P173=3.39482419503876e-05, P174=3.41799504787769e-05, P175=3.307209972507064e-05, P176=4.1211829081963594e-05, P177=3.33726617101956e-05, P178=2.0464076866152477e-05, P179=2.4974328385932025e-06),Row( P0=-454.26801049740214, P1=-6.504376786174779, P2=-34.916603737650945, P3=-9.36937006220671, P4=-14.689364203176174, P5=-11.100418712981599, P6=-1.825271798513564, P7=5.992723810226699, P8=12.945029751770557, P9=12.801730112404545, P10=3.6996679506284345, P11=2.2559712331234003, P12=-14.193072197369185, P13=-8.51534554159502, P14=0.830229894732077, P15=8.073289259745051, P16=4.22481924891827, P17=0.23712442869086114, P18=-3.234472763968545, P19=0.4197788162901975, P20=2.7287295072453412, P21=1.7984304010084036, P22=0.3039436513760941, P23=1.5205591104731104, P24=-1.8909216343510566, P25=-1.5060340223844717, P26=-0.6183264148307343, P27=3.0234224935396425, P28=-1.4147792504123347, P29=2.4962021143263717, P30=-1.5356292144308519, P31=0.9699693872631352, P32=-0.15644208747194993, P33=1.1623860636332126, P34=-2.246574231526122, P35=-1.4831282493375426, P36=0.39625994276705906, P37=3.564602683202294, P38=-2.7154679056105895, P39=-0.5828225454492016, P40=0.6667626119051567, P41=0.6619769538010158, P42=0.6075052755955183, P43=0.5016169387286343, P44=0.47362052868476984, P45=0.47932851260540765, P46=0.5113408952609743, P47=0.5279261989708973, P48=0.5717010325067376, P49=0.6451326603476777, P50=0.6738675155346331, P51=0.6638771399809735, P52=0.0047920868908096735, P53=0.006059262848418907, P54=0.0024325664068174393, P55=0.0007175643287690012, P56=0.0005540208627574139, P57=0.00042599705758908244, P58=0.00029938148743862726, P59=0.00011606049587736403, P60=7.882686288561838e-05, P61=7.14507019031544e-05, P62=7.922469936127229e-05, P63=6.452264832512918e-05, P64=4.299248203656776e-05, P65=8.306242997333417e-05, P66=0.00012401958325389994, P67=7.852887753410006e-05, P68=8.86469284052713e-05, P69=0.00014982058347425405, P70=0.00013843398105823525, P71=0.00022372122798330378, P72=0.0004665173914098184, P73=0.0025243318143575834, P74=0.022939304359894774, P75=0.043075583101821205, P76=0.03454913774396741, P77=0.04148367940974734, P78=0.11091376795551773, P79=0.28166840259260484, P80=0.5803057411542185, P81=0.60019242605803, P82=0.5661757882004587, P83=0.46050789590827934, P84=0.39684998161486884, P85=0.5590559853628873, P86=1.2009335122601803, P87=2.677206467257543, P88=1.0250743022167261, P89=0.25239110757191824, P90=0.07836610052208756, P91=0.028714995894683733, P92=0.004768789148973524, P93=0.0013672624580046823, P94=0.0015988271020386986, P95=0.001964182745292203, P96=0.001574768304302766, P97=0.0071762219369881294, P98=0.05131863622857542, P99=0.742632898741332, P100=1.772107518435858, P101=1.2960885303071328, P102=0.4337712196693227, P103=0.23558633239937404, P104=0.2128670191841111, P105=2.430991721191938, P106=1.431245501503273, P107=2.7273063112255023, P108=24.671983016266992, P109=4.441317222570354, P110=0.01975595715118426, P111=0.04646443854476134, P112=0.1391598864109423, P113=0.08068169637692246, P114=0.06216383636431, P115=0.11925787475642617, P116=0.06781918861620452, P117=0.21974503263908401, P118=0.9037301669630893, P119=1.2678874886532845, P120=0.6831651845129965, P121=1.1730697119690778, P122=0.5369975970043854, P123=0.13601679522404958, P124=0.10869930549304543, P125=0.05974956239669706, P126=0.07308072934375039, P127=0.04188772608068596, P128=0.02331874413649166, P129=0.07224734701106968, P130=0.24466223194973233, P131=0.138948012371507, P132=0.04706759051761017, P133=0.04138808264988788, P134=0.010168733311978408, P135=0.008783719058115558, P136=0.029901892369096877, P137=0.020696193992406742, P138=0.01965787881550688, P139=0.019835102807281086, P140=0.030906294140393857, P141=0.06466226514490842, P142=0.09271405400372011, P143=0.054809242323557196, P144=0.009241101594108963, P145=0.015525770259137534, P146=0.0215336722721637, P147=0.016959327272852497, P148=0.009989773780538776, P149=0.009378407846433848, P150=0.005948635266698837, P151=0.00874626773700129, P152=0.007734880428963524, P153=0.004899744898450709, P154=0.0034917431810612674, P155=0.0013204861037596721, P156=0.0007906045768081867, P157=0.007707849842550527, P158=0.004623113070173264, P159=0.0007993852879304553, P160=0.0008782310245114368, P161=0.0008951284708010562, P162=0.0010741007477560456, P163=0.0011290048904973723, P164=0.001358531210524215, P165=0.0015466705866266138, P166=0.0014806508500113432, P167=0.002961585221074584, P168=0.00178170193561184, P169=0.0008042913187335546, P170=0.0010667278309802748, P171=0.0010318069201359684, P172=0.0009258416684889303, P173=0.0011816168270410079, P174=0.0012145663079507487, P175=0.0014821297406025149, P176=0.0012733635305177049, P177=0.0018094108800663607, P178=0.0009651906782881298, P179=7.062064124098112e-05),Row( P0=-606.0007696400229, P1=34.47288715604149, P2=-29.01223304414108, P3=-12.731431295677586, P4=-11.355370760301447, P5=-13.132004030807522, P6=-8.126252467272215, P7=-10.217766631000192, P8=-1.8507647765083566, P9=-10.676354729054554, P10=-9.959204984965137, P11=4.219896864952879, P12=6.369396299565895, P13=8.715887061675925, P14=15.357780735497144, P15=6.583114375868858, P16=1.4693565629713665, P17=-4.657831077670447, P18=-2.2241952469302877, P19=1.0944977485143754, P20=-1.6930131720469752, P21=0.532342971644472, P22=-2.245092718629408, P23=-4.566555141816536, P24=-0.1132753193696991, P25=-1.5595742164496609, P26=-0.7755570249558129, P27=0.5600087517446585, P28=-4.728459347946404, P29=1.6634818756618353, P30=-1.1371276737235831, P31=0.6481929744911998, P32=-1.8695899835109657, P33=0.059146837505462097, P34=0.2423999561844599, P35=2.6163984565407015, P36=-2.4552557321257913, P37=0.1964601000639485, P38=-1.5456831422593047, P39=1.2885083458167015, P40=0.6424487022127042, P41=0.6495000160426126, P42=0.655475939223234, P43=0.7050413828266832, P44=0.7201642667677717, P45=0.6787868388167309, P46=0.6448750404185931, P47=0.616913663923035, P48=0.61771487625035, P49=0.6271038364271128, P50=0.6188079604023885, P51=0.6074676195018256, P52=0.0005307705264601349, P53=0.0002915369375356976, P54=5.786012744970472e-05, P55=4.322820085206147e-05, P56=6.0742527389869435e-05, P57=7.095259910481779e-05, P58=4.8606555719758075e-05, P59=4.2257121058257025e-05, P60=4.4545714836454605e-05, P61=3.1879335660445474e-05, P62=6.107867265325868e-05, P63=0.00014594999219766906, P64=0.0008798628303261383, P65=0.0068505889189804684, P66=0.024873398567097707, P67=0.0404183590081164, P68=0.048996102376306735, P69=0.1250612851319395, P70=0.185810228910416, P71=0.16178561985470802, P72=0.18069924376460947, P73=0.1377175857004503, P74=0.14940630938907867, P75=0.12306130523278984, P76=0.05708367085724952, P77=0.018624052540477096, P78=0.0034657871070046292, P79=0.001548309708033457, P80=0.001302474036408014, P81=0.0026490548936924808, P82=0.009752660035235085, P83=0.010122936878911214, P84=0.01338995704560703, P85=0.028173288708430944, P86=0.04153662496791943, P87=0.07278392624403518, P88=0.24307186874512304, P89=0.3315383168051156, P90=0.3415413303489654, P91=0.3272415613433597, P92=0.07260705544331512, P93=0.0314864079091252, P94=0.09312307402283167, P95=0.1188999889995629, P96=0.03218842158955981, P97=0.04645941139057056, P98=0.06604004981188329, P99=0.042535119826968296, P100=0.03081708442175451, P101=0.021385958934318616, P102=0.025877433919026034, P103=0.04057912644921735, P104=0.022016192179305286, P105=0.03984481395794586, P106=0.030823638929107527, P107=0.008809001865389956, P108=0.004661872887386599, P109=0.016695633899939107, P110=0.03241029788428494, P111=0.03509089413327983, P112=0.01823756839125626, P113=0.007420952676273328, P114=0.014004436392922833, P115=0.0231311525104933, P116=0.01681101512103093, P117=0.013168648441691239, P118=0.01301049054291431, P119=0.01111519620268445, P120=0.007744523821114137, P121=0.009673284699458004, P122=0.005644511785170424, P123=0.002634613475582342, P124=0.0024907332457886565, P125=0.0021147288956401477, P126=0.001421246835851142, P127=0.0013625435706692958, P128=0.0010206344236565077, P129=0.0010686940759372075, P130=0.0006034117651810527, P131=0.000367762150585543, P132=0.0003612989402524997, P133=0.00010747360017930763, P134=0.00013873311213691538, P135=9.096780056701559e-05, P136=0.00011314580515601382, P137=0.0002330669161746926, P138=0.00032459993872357686, P139=0.00029139043116676043, P140=0.00040768657839023283, P141=0.0002878100220526252, P142=0.00027400837914564593, P143=0.00021334485369568746, P144=0.00019828831643267932, P145=0.0001834955029775874, P146=0.00020159142549322278, P147=0.00024144572516648577, P148=0.00032712171324433153, P149=0.00021202255729997063, P150=0.00014811490115322216, P151=7.606226808003848e-05, P152=5.024514065435516e-05, P153=5.7574299939853575e-05, P154=4.885038530356944e-05, P155=0.00011844376375511218, P156=0.00014387538700590306, P157=0.00016291766218865618, P158=0.0001327358494002784, P159=6.570842872674063e-05, P160=8.315482550423236e-05, P161=0.00012060838083856309, P162=0.00011317788292132384, P163=9.619383991410501e-05, P164=9.249179983212183e-05, P165=0.00010189418706404664, P166=5.9952630931961154e-05, P167=3.8432701562147834e-05, P168=8.936197210626586e-05, P169=7.906511163126268e-05, P170=4.221110586273002e-05, P171=4.6031835775861517e-05, P172=6.258151930093928e-05, P173=9.774369134052919e-05, P174=0.0001260855061758369, P175=0.00011343729707873065, P176=0.00014634680884416458, P177=0.0001180025775135667, P178=5.724605559900563e-05, P179=8.486744175367518e-06),Row( P0=-686.1655635477723, P1=43.97221169170277, P2=-19.09061743802155, P3=-4.378439990849181, P4=-16.085764449030556, P5=-16.043402137208204, P6=-10.972595233170235, P7=-8.402578752701302, P8=-2.7022522789473764, P9=-13.992075282507647, P10=-10.953232995653545, P11=-8.358223475374855, P12=-7.973804120619942, P13=-4.513713600492695, P14=-1.1060787290508973, P15=4.424468556071367, P16=-0.8331219075311117, P17=3.18661253767908, P18=5.49585344863962, P19=2.5115319958940083, P20=3.072073776747425, P21=5.69999934256806, P22=3.248451699512615, P23=2.587761865212872, P24=2.032619101151897, P25=-0.9927348536000661, P26=-1.325892133282846, P27=-0.35065488208877754, P28=-2.2186928171091314, P29=0.748054215164814, P30=-1.9791475631429647, P31=-2.1631336351888515, P32=-1.648064364240088, P33=-0.4466411894987748, P34=-2.7690453897437575, P35=-2.3063237576135673, P36=-5.306456245748541, P37=0.2062687816753886, P38=-0.36666066159407895, P39=0.4894868974812164, P40=0.6781291638804302, P41=0.6373361016462471, P42=0.6198962810260927, P43=0.6373561238837221, P44=0.6730947660578376, P45=0.6230556999347711, P46=0.6253253824877174, P47=0.6087470139320257, P48=0.6070802851459994, P49=0.6007251274567612, P50=0.5967382186163988, P51=0.6279437154026828, P52=2.8658516606334193e-07, P53=1.5480500849543487e-06, P54=5.905993219559278e-06, P55=9.833275382611966e-06, P56=1.7428660810873184e-05, P57=2.2870631920451346e-05, P58=1.2441983441480942e-05, P59=1.107627995119436e-05, P60=0.00041193580574221865, P61=0.004528877437324709, P62=0.015522887375318945, P63=0.021405042666811484, P64=0.02144217488617786, P65=0.05510959557845722, P66=0.07774580882500245, P67=0.06368237841362405, P68=0.049348168587654104, P69=0.04745409036201014, P70=0.03512105659281296, P71=0.01858068003129408, P72=0.017700669772590348, P73=0.013410804960783194, P74=0.011071114153473487, P75=0.014593854809414176, P76=0.010860420600676532, P77=0.01607117822969311, P78=0.024230547978301195, P79=0.022386052222696923, P80=0.010094711788152704, P81=0.005541884616249021, P82=0.004531707705134823, P83=0.004382815714560826, P84=0.016067911832652052, P85=0.026287671328245433, P86=0.012273341694296122, P87=0.01466572973780065, P88=0.011183316531821499, P89=0.011321412365967006, P90=0.023377873617914248, P91=0.034345272759375114, P92=0.012677038384352631, P93=0.003985595364018871, P94=0.0046361318107701315, P95=0.003953139109603365, P96=0.0022023237396294865, P97=0.0035666637164882883, P98=0.003761021494098586, P99=0.0009328033269898062, P100=0.000815965275709548, P101=0.0005483648280720997, P102=0.00042827208998561704, P103=0.0003461081949002918, P104=0.00036634077393565124, P105=0.0015735281855252321, P106=0.0012761319458129594, P107=0.000515548411931515, P108=0.00018245348944840982, P109=0.0004200324542900593, P110=0.0007441367170135059, P111=0.0009893230970766077, P112=0.002170439334484363, P113=0.0011480930476029164, P114=0.00028866662027247865, P115=0.000561881165335203, P116=0.00043861654212932913, P117=0.0004562621225576302, P118=0.0005162959964558611, P119=0.0008050104718988994, P120=0.0010624617210050077, P121=0.0007110076268762969, P122=0.0008055390627577622, P123=0.0006028825777385617, P124=0.0005251321362341457, P125=0.0004232887641415645, P126=0.00012182023125547988, P127=0.0002565585063674509, P128=0.0001120935994348631, P129=7.688371388473761e-05, P130=7.823692072724453e-05, P131=5.558914288684224e-05, P132=4.311035662783174e-05, P133=3.320952171794269e-05, P134=4.003000438572281e-05, P135=7.797547655993614e-05, P136=0.00012692343678761015, P137=8.313197616402886e-05, P138=9.13875673648947e-05, P139=0.00017441702012469925, P140=0.0001247421717437823, P141=5.8416791248693056e-05, P142=3.6818966690632516e-05, P143=1.181881000286699e-05, P144=1.3329192694675138e-05, P145=1.2355518673396202e-05, P146=1.873892434816133e-05, P147=2.6461349100328607e-05, P148=2.734733765241857e-05, P149=2.227783326114985e-05, P150=3.177506319109868e-05, P151=3.3080026422786405e-05, P152=2.008761068677154e-05, P153=1.3945845960799851e-05, P154=1.474127307816671e-05, P155=2.099338551485378e-05, P156=4.212425845968426e-05, P157=3.472611872403777e-05, P158=2.684118298832209e-05, P159=5.234100235535979e-05, P160=7.98577411739019e-05, P161=5.803070566759458e-05, P162=5.966174301240633e-05, P163=5.398850086259781e-05, P164=6.0060066488693306e-05, P165=9.350692734204434e-05, P166=7.479169018917344e-05, P167=6.365015333513033e-05, P168=8.510030771094719e-05, P169=0.00024139392565355095, P170=0.000273829543193747, P171=0.0002437460849137007, P172=0.00010911825271580466, P173=0.00012443678659181814, P174=0.0001187734350922027, P175=0.00010693883868020848, P176=6.843164099663127e-05, P177=6.981164955271968e-05, P178=3.6748729544745817e-05, P179=3.1770936279860983e-06),Row( P0=-500.4617113217274, P1=-0.31610517158133167, P2=-42.55867194840205, P3=-5.93975145288649, P4=-19.714291415152232, P5=-16.027426477626925, P6=-1.8897496131794693, P7=5.764782533515741, P8=11.022832475961389, P9=11.970466123826794, P10=2.7271498329902384, P11=6.688871154654779, P12=-10.886320835169787, P13=-10.06672344770935, P14=2.0067529187152275, P15=4.339606020683258, P16=3.8395698539111045, P17=4.721608889527866, P18=-1.067386406917998, P19=0.93428006231131, P20=1.3826875556542297, P21=2.2849183585355903, P22=-3.523917848608112, P23=0.3548705439499905, P24=2.923298698566299, P25=0.3379227580120883, P26=-2.3372820074694456, P27=0.640647518628456, P28=-2.010046013540225, P29=2.279835247171898, P30=-2.7863751548008713, P31=1.199079422631525, P32=0.3926291240004173, P33=0.23308243662539282, P34=-1.14432839240926, P35=1.958718176371852, P36=-2.4674310809903064, P37=-0.6135225753172272, P38=-1.4336768530301178, P39=1.035335571220721, P40=0.7292032934024266, P41=0.6762240335014644, P42=0.6007232113965492, P43=0.5486288511642274, P44=0.5236569107864282, P45=0.5061187157403793, P46=0.5222801216026758, P47=0.5780003492516067, P48=0.579262255833977, P49=0.6288333246838295, P50=0.6853566831979048, P51=0.714083686533025, P52=0.0005105276485628118, P53=0.0017671934005070386, P54=0.001864669037535003, P55=0.0005053711787693262, P56=0.00048507716924942355, P57=0.00023622448804037587, P58=5.8613092974914134e-05, P59=6.169541748347747e-05, P60=9.723345780110284e-05, P61=0.00018333357236029006, P62=0.0002816462023068645, P63=0.0003602209098451826, P64=0.00025807863977509815, P65=0.00027975239544191085, P66=0.0003489992071762537, P67=0.00035228609197289056, P68=0.0005360350915946025, P69=0.0013347830122211893, P70=0.0019989527036352464, P71=0.001536092740962481, P72=0.004693167026778826, P73=0.01298479224285419, P74=0.047172802204178806, P75=0.08828360438897555, P76=0.1239122996911707, P77=0.23641612260789543, P78=0.3007200875577788, P79=0.5155614441303156, P80=0.9477481132097517, P81=0.9837811231671373, P82=1.502586899352903, P83=1.440512489858148, P84=0.719403599568985, P85=0.36953704812105687, P86=0.237366273099059, P87=0.3547179317207513, P88=0.6162152873447241, P89=1.5670105916973398, P90=0.9107815201552328, P91=0.06250526133608451, P92=0.011064422533879376, P93=0.0022430963401126338, P94=0.01055400931861157, P95=0.08458553001279141, P96=0.15795670299070388, P97=0.32811427300661433, P98=0.4615678523082186, P99=1.1857330976063352, P100=1.1183235806639649, P101=0.5588210222332148, P102=0.26197757543414574, P103=0.2908603732322849, P104=0.628020328676822, P105=0.9124730794684461, P106=0.73988505215889, P107=0.5098470971263447, P108=0.14348584075892837, P109=0.2092067945752204, P110=0.12915575589802314, P111=0.10627434541486055, P112=0.08952388645237216, P113=0.07569355546254598, P114=0.0839423265840668, P115=0.08039453191897253, P116=0.2885396866276654, P117=1.1357105602661015, P118=0.2677933838517182, P119=0.14299221640927057, P120=0.3107166895628107, P121=0.4806266123909128, P122=0.4445160760063163, P123=0.16393803064078658, P124=0.13007864103063496, P125=0.09238723868298584, P126=0.03392624777018487, P127=0.10555574456405752, P128=0.08343463002841381, P129=0.06063504803495898, P130=0.028398127493821923, P131=0.025960712808667057, P132=0.028536381613669077, P133=0.04145477490423632, P134=0.024324200423274465, P135=0.007308031418129865, P136=0.0109742310574733, P137=0.07965124631461763, P138=0.08774651962470481, P139=0.03025844881490705, P140=0.019832969072812297, P141=0.04022286565887434, P142=0.04222631809267313, P143=0.03245290589277586, P144=0.015209084829042246, P145=0.03930696103143879, P146=0.029362604853541897, P147=0.01774488444819651, P148=0.011025747106800033, P149=0.00632005355121859, P150=0.00368601507026728, P151=0.004122164695156838, P152=0.0011169346251481318, P153=0.00037773398697018716, P154=0.00031828214305517065, P155=0.0006554560864032849, P156=0.0010799958582366534, P157=0.0007735574195609224, P158=0.0007107846655034986, P159=0.0006166126762597049, P160=0.0008448531636872082, P161=0.0006867764915787113, P162=0.0007274232826921329, P163=0.001017039502382322, P164=0.0007620825991754054, P165=0.0009255563883585854, P166=0.0006333992795865326, P167=0.0006605980329041712, P168=0.0006867117704278657, P169=0.0005277248553794545, P170=0.00041940806513699543, P171=0.00036354327228559704, P172=0.0003560432577885693, P173=0.0004391752516450232, P174=0.0007576682193026983, P175=0.0008098209106142046, P176=0.0008616540080283244, P177=0.0009655240435440306, P178=0.000517938358943279, P179=3.420848014821997e-05),Row( P0=-566.8196855732396, P1=30.080433364957415, P2=-25.357188340932556, P3=-4.139084424960482, P4=-18.818236810501155, P5=-13.863463792213782, P6=-8.18699751430814, P7=-5.815109869265161, P8=-8.843914948506358, P9=-11.836064486947526, P10=-8.318307908239422, P11=2.275361367547093, P12=-3.2221370414195083, P13=6.886787407470153, P14=7.767309615081235, P15=2.5302043665890483, P16=1.7165513817752796, P17=-0.9613885449945491, P18=-0.7350882743934762, P19=-1.0013770846144636, P20=-0.2049974487455252, P21=1.3176223869459982, P22=0.049915587747039025, P23=2.563096275604732, P24=3.2469888634470463, P25=-1.3708872359324225, P26=-4.212579454620305, P27=-3.8646718206002997, P28=-1.0196811304905113, P29=4.484897253044224, P30=-2.0197664901491765, P31=-0.7003966294520586, P32=-0.6744013923312786, P33=0.4359177541121658, P34=-1.0990865263553165, P35=-2.760928835151166, P36=-1.2842137244354128, P37=1.0872958610846715, P38=0.23624359141495255, P39=-0.2792508871836269, P40=0.615995070749719, P41=0.619915861842159, P42=0.5754533326998579, P43=0.5105342076097962, P44=0.5601717824409458, P45=0.5615263601216615, P46=0.5568132500661735, P47=0.5040341807962051, P48=0.4789993231554391, P49=0.4778565827073453, P50=0.49872243742957273, P51=0.5625537597789368, P52=8.844042732352208e-06, P53=1.0598639795413018e-05, P54=1.723590879099929e-05, P55=2.7089261906063278e-05, P56=5.169048948670119e-05, P57=7.403629271256551e-05, P58=4.07396009497983e-05, P59=3.698788164199763e-05, P60=6.663956886689578e-05, P61=0.0001885563234941618, P62=0.0049401754764938745, P63=0.009794372459494813, P64=0.009632392833776496, P65=0.03522036144184438, P66=0.1573698809945605, P67=0.36568544920416474, P68=0.2075100061043996, P69=0.14454771529425725, P70=0.10196435871737249, P71=0.053879950299543296, P72=0.11618735370624146, P73=0.10789619036326434, P74=0.1270067016153047, P75=0.11564969551466506, P76=0.04657562401035752, P77=0.0182769902273761, P78=0.01298211333199045, P79=0.008846189695469442, P80=0.005187622945819857, P81=0.004337470979729694, P82=0.008583813201691503, P83=0.03732917721785871, P84=0.08683231879020316, P85=0.25712093710564193, P86=0.35449974396288086, P87=0.47162051128010485, P88=0.6023236413522136, P89=0.1992156506447636, P90=0.09722687611336983, P91=0.14452904620953788, P92=0.12863981189941448, P93=0.05099581405271972, P94=0.08055732288872779, P95=0.08875052823426695, P96=0.1359401191931965, P97=0.15305465664900747, P98=0.07126592843932317, P99=0.02301080438502777, P100=0.01962178602005881, P101=0.010701371827251704, P102=0.01360636230846943, P103=0.14859698264236176, P104=0.27469642926041576, P105=0.2074957623802965, P106=0.05787902521869322, P107=0.004787123164345825, P108=0.007045274070014066, P109=0.007949890085118379, P110=0.002766548363910304, P111=0.006899654830149126, P112=0.09369049135023251, P113=0.1331487823529899, P114=0.07101483168587393, P115=0.023206299276807205, P116=0.005504729092757486, P117=0.01041085015612274, P118=0.014003388528301358, P119=0.011179963906225174, P120=0.012450493071338404, P121=0.014838720999985894, P122=0.06318579597032871, P123=0.042316113384112924, P124=0.00658693819056753, P125=0.008836246393191322, P126=0.011363155669875218, P127=0.012562292773269625, P128=0.0060125663578934575, P129=0.004075262104090398, P130=0.004357134433130336, P131=0.006701433620485142, P132=0.007495803677453873, P133=0.0017295081722837598, P134=0.004146290293157512, P135=0.005484873296641397, P136=0.0014846746122911105, P137=0.000839114588896448, P138=0.0010504694518589468, P139=0.0021280986252992075, P140=0.002111158144164989, P141=0.003485766503477411, P142=0.0012604492146444733, P143=0.0012501330201310896, P144=0.0008760694464135746, P145=0.0013648612880507482, P146=0.0012888771346713304, P147=0.002032543403267393, P148=0.0020573562171509576, P149=0.0028598918042858613, P150=0.0025480173959470625, P151=0.0018220401305508128, P152=0.0004141646546288392, P153=0.00020181932573606822, P154=0.00017598169289649674, P155=0.0006132880795449459, P156=0.0008571255028144738, P157=0.0010255991847900148, P158=0.0003710742371978622, P159=0.0005595764364111048, P160=0.0007327611518209491, P161=0.00042055070359543696, P162=0.00038900695497172936, P163=0.00034376196824315795, P164=0.0003207317569488809, P165=0.0002591320575620251, P166=0.00035288903611446786, P167=0.00038039844601686385, P168=0.0003911851305179705, P169=0.00025008730833989037, P170=0.00018329298015467348, P171=0.0001863138439153728, P172=0.00019750396173121997, P173=0.00029036608326697813, P174=0.00027863680544563127, P175=0.0002352041372780122, P176=0.00026345391269122774, P177=0.0001766178077089638, P178=0.00011782000053268308, P179=2.6951010866467573e-05),Row( P0=-610.4207961271634, P1=32.62580383467082, P2=-19.450164209416478, P3=-5.229016142227596, P4=-18.964869121220435, P5=-15.66033881832345, P6=-6.119087336266425, P7=-3.87262920761624, P8=-7.891437505259148, P9=-10.224157548246243, P10=-10.221652320135025, P11=-2.0997791156558607, P12=-9.737709917092399, P13=-2.6177406571069715, P14=1.8957124938952807, P15=2.532130515694727, P16=6.338359689439694, P17=6.135216933786852, P18=2.81379010920161, P19=2.703545130427076, P20=0.42416114820333534, P21=0.9832393852835585, P22=1.9824665198316993, P23=0.5982928457226973, P24=-0.7866670763758804, P25=-4.407043292555699, P26=-3.688794558377487, P27=-2.126313604635336, P28=-0.31536528243188133, P29=3.1310121186446214, P30=-1.9756689400374754, P31=1.0756174505299205, P32=1.3467270123665522, P33=3.743669593791397, P34=0.76363873124785, P35=-1.714238964999962, P36=-3.625194582587507, P37=0.12767381255464663, P38=-0.13763183558592582, P39=0.7509808489724548, P40=0.5788662474396463, P41=0.5309240652919512, P42=0.48797532398700644, P43=0.4742120351325745, P44=0.48545814880686633, P45=0.4511741584190545, P46=0.45011640119279384, P47=0.4815518106910197, P48=0.505938035753674, P49=0.4716405139793528, P50=0.4775547771864196, P51=0.559526483404018, P52=2.816618260174184e-06, P53=3.3609735282194258e-06, P54=2.734736595142776e-06, P55=7.827079155978797e-06, P56=1.76628975131651e-05, P57=3.871669563144178e-05, P58=3.234635947962062e-05, P59=8.073299557720068e-05, P60=0.0003723932078162489, P61=0.001953242093402893, P62=0.006067175341273744, P63=0.01607236466120984, P64=0.026716872500953788, P65=0.030356619656593746, P66=0.0672723499055474, P67=0.11477670147874025, P68=0.07064217851043676, P69=0.06664314058829479, P70=0.03443338702311177, P71=0.006114332045818062, P72=0.001667137913458665, P73=0.0015921553222224274, P74=0.003082217790369348, P75=0.005768594576777841, P76=0.02236986464701099, P77=0.04747313818892186, P78=0.05129304823959335, P79=0.03065205472808087, P80=0.020509181626638347, P81=0.01886382942286426, P82=0.04446264053234496, P83=0.12630165233511692, P84=0.17057682297061227, P85=0.254872331672514, P86=0.22426187272132153, P87=0.21742033909563357, P88=0.1588185535736005, P89=0.04787748685961375, P90=0.04055297789412237, P91=0.030048908898653404, P92=0.016861490891612232, P93=0.012497772510974035, P94=0.012681583036269463, P95=0.03443701923775088, P96=0.0534379675856817, P97=0.06238024547399163, P98=0.11262920680268262, P99=0.06684454385974122, P100=0.012111295932076027, P101=0.003154735349817786, P102=0.0010341806981942852, P103=0.0009085187891328197, P104=0.00808355052261262, P105=0.029981397484098858, P106=0.026271291662862687, P107=0.004884478193753393, P108=0.000613603070770248, P109=0.00028502037586635885, P110=0.0006201267278019676, P111=0.004175713749970438, P112=0.0035103278254931176, P113=0.002396175255951793, P114=0.003089897655649633, P115=0.0013031718089732312, P116=0.006185214645499985, P117=0.0099964320663924, P118=0.002158869910377782, P119=0.0037178278442617004, P120=0.007882848285077834, P121=0.005431052697681598, P122=0.00186893329958504, P123=0.0044188466983772455, P124=0.007535773680252339, P125=0.002370710649273461, P126=0.002129190797615382, P127=0.004500991745961276, P128=0.0015712618156292512, P129=0.0022040043170606987, P130=0.0029957081071423407, P131=0.00181190535242825, P132=0.002749019930503225, P133=0.004801245537169218, P134=0.0021220213802499127, P135=0.0010785935645194814, P136=0.0004305402094826541, P137=0.00041068627178202533, P138=0.0004190110803652569, P139=0.0008329959147346551, P140=0.0012313820012046339, P141=0.0016672560568930367, P142=0.002420456810659086, P143=0.0026822542696825044, P144=0.001401832266255855, P145=0.001188914679745117, P146=0.0007630802127696932, P147=0.0007453863667694502, P148=0.0007014823360774928, P149=0.0005799310196937032, P150=0.0006007605270076185, P151=0.0006312536132396652, P152=0.00014722122073434937, P153=6.52110430346888e-05, P154=0.00010940192477641119, P155=0.00019224789053798205, P156=0.0005030752175098343, P157=0.0005940720902831198, P158=0.0004671157205845633, P159=0.00040511020668047165, P160=0.000800554896298528, P161=0.0007330888310112616, P162=0.00045606085573107443, P163=0.0004776163594804982, P164=0.0008108022623447002, P165=0.0011451303564593083, P166=0.001570955533507265, P167=0.0016861890087628564, P168=0.0008716829862758407, P169=0.000576232430981533, P170=0.00043795022500040946, P171=0.001723776378102532, P172=0.0022261901350063822, P173=0.0013406263482159061, P174=0.0006798867067405174, P175=0.0004894571199842089, P176=0.000535314960007976, P177=0.0005436537302900439, P178=0.00031946678684226693, P179=3.536782936622017e-05),Row( P0=-625.7050250676758, P1=50.91032304717572, P2=-16.38798475555719, P3=-8.013654864990817, P4=-18.228385417421418, P5=-8.736751336436608, P6=-11.206103557248639, P7=-2.154503967190342, P8=-5.158755750683094, P9=-11.984001557158882, P10=-8.12518522418118, P11=-2.19209832780729, P12=-10.070032550834933, P13=-4.6369028988942675, P14=-2.1406631671672645, P15=-6.506889630865859, P16=-3.1229573300562405, P17=-1.3500425401557823, P18=0.9987274434025096, P19=3.330552136775593, P20=2.600350358202681, P21=4.066234805514306, P22=2.716374591634444, P23=3.2471053446625615, P24=-1.7930023710424952, P25=-4.075498590251657, P26=-4.6741279673135665, P27=-3.0437891189970165, P28=-0.7159708466745601, P29=0.3327726776613108, P30=-1.6541389719959105, P31=-0.2433728632148166, P32=-1.3732038449764192, P33=-0.18086054436749405, P34=-1.3467379273266438, P35=-0.35827796894482083, P36=-1.0151535873917454, P37=-0.4251121998142154, P38=-2.3483712572914275, P39=0.2705232992558775, P40=0.5134833445591427, P41=0.5352178652014534, P42=0.4968957883620754, P43=0.4674303749953319, P44=0.4656807667834329, P45=0.4690961916604153, P46=0.5245162515016908, P47=0.5442341059266971, P48=0.5314589440628001, P49=0.5179959520053917, P50=0.486559817949121, P51=0.48334309280463894, P52=2.4552193418496515e-06, P53=4.490123440931087e-06, P54=9.110331927924742e-06, P55=3.487922897460981e-05, P56=9.126847733234664e-05, P57=0.00021556564485704835, P58=0.00020776259371225818, P59=0.00029054434427811004, P60=0.0005472886379070691, P61=0.0016938236366599003, P62=0.013194437062893322, P63=0.03531035116569984, P64=0.02355920376948934, P65=0.033139510593178774, P66=0.08206818903477002, P67=0.0815885409175826, P68=0.02169689735391287, P69=0.006099512487532113, P70=0.0028913951320610146, P71=0.0014460115797562656, P72=0.002030460877159087, P73=0.0031037774497677004, P74=0.017687322114534422, P75=0.0429747842455881, P76=0.04836562493883616, P77=0.05112968558362024, P78=0.0554136567428054, P79=0.04508364738626347, P80=0.041428390481982616, P81=0.05971367330546862, P82=0.08626220674677476, P83=0.05994825078252908, P84=0.03285244405392009, P85=0.030105061708761382, P86=0.027766570733660546, P87=0.03345305113540425, P88=0.04086539825851928, P89=0.03068269254450668, P90=0.023757439028466866, P91=0.017934801164903022, P92=0.00857721979108205, P93=0.004691489984492532, P94=0.011372000131235084, P95=0.017486511980844204, P96=0.008736922650673157, P97=0.026194933381443106, P98=0.03219117260535376, P99=0.009276091280981776, P100=0.002577137755466659, P101=0.0017293777568801064, P102=0.0025279893189364367, P103=0.0018988870759394717, P104=0.003905780311908747, P105=0.01342809056978728, P106=0.02859599956340521, P107=0.017794137995343403, P108=0.008918435176444797, P109=0.0030191494683189045, P110=0.00299246894833593, P111=0.0012446123499263646, P112=0.0010938302288618009, P113=0.002726105136041903, P114=0.0025973647011873907, P115=0.00244552319375478, P116=0.0022435029985082297, P117=0.0024449189847948064, P118=0.0010084544003039402, P119=0.0007738251161132925, P120=0.0004966900535037052, P121=0.00045678186326508806, P122=0.0006033411031992088, P123=0.0009599784866049472, P124=0.0006568139975805863, P125=0.0007541987366717727, P126=0.0009896520435561254, P127=0.00017564496480775263, P128=0.0003157629593377665, P129=0.0005629879104531372, P130=0.0002364615727180616, P131=0.00041845434289117245, P132=0.00046194958251218, P133=0.00018678762330280867, P134=0.00017595323466619515, P135=0.00011043586952730954, P136=6.69894663317762e-05, P137=6.876042807020046e-05, P138=7.921075056413653e-05, P139=7.670015362542708e-05, P140=0.00013756640382762286, P141=0.00019044407183626885, P142=0.0002316105790393229, P143=0.00019405607238983852, P144=0.00023750899719130367, P145=0.00021008634892917996, P146=0.0003796291120337331, P147=0.0008142878057247715, P148=0.0006736092405182477, P149=0.001037051093049112, P150=0.00073734570598267, P151=0.0005001580059376013, P152=0.00036970298911115324, P153=0.00025102490400033404, P154=0.0002828978308632946, P155=0.0002650437383061883, P156=0.0002116822925322659, P157=0.00033737534737441846, P158=0.00027452087753636426, P159=0.00042619899693879874, P160=0.00038484035283516013, P161=0.0002831379292891056, P162=0.00027337803661099533, P163=0.00026887909683220184, P164=0.0003797735821677494, P165=0.000190072387241697, P166=0.0001786230293513787, P167=0.0003894810947558685, P168=0.0003592904302973074, P169=0.0002489954828779099, P170=0.00018825432779339714, P171=0.00010081043255984401, P172=8.576375598685908e-05, P173=0.00011013457347044866, P174=0.0001252513774069389, P175=0.00019022611997837735, P176=0.00014829075606911848, P177=9.90868628504353e-05, P178=2.544409901866485e-05, P179=2.5382000834014916e-06),Row( P0=-440.4780752174631, P1=3.4943862633904366, P2=-40.16608195450013, P3=-10.248321860533347, P4=-28.287439195183918, P5=-19.276230475878823, P6=-4.459851832203469, P7=-0.048897007431636955, P8=8.971116748263881, P9=6.306623251606118, P10=5.563669950740645, P11=14.825020148992985, P12=-6.973132367106634, P13=2.890578074300057, P14=4.58352783948974, P15=-6.833928441147703, P16=-2.3006235346281887, P17=1.057351131358459, P18=-1.8014297403282198, P19=-2.7094607316958523, P20=1.0378264444763492, P21=4.944050180775696, P22=3.2186770661194357, P23=2.2478933255098577, P24=-1.2653962751429289, P25=-0.6889790332845402, P26=-0.6349709221744515, P27=2.286390175766836, P28=-3.5959419382967504, P29=3.070047461979662, P30=-0.18373596361400618, P31=1.9874915163813938, P32=-2.344745055023341, P33=-0.24490838042534596, P34=0.12349425835256878, P35=0.8466873475653836, P36=-0.8677749396361721, P37=-0.8470466036907472, P38=-1.1687319504594695, P39=2.4668350260706213, P40=0.6803440813804797, P41=0.6860858411564095, P42=0.6172389009543992, P43=0.5581991154103944, P44=0.5442860434867992, P45=0.5234423206379668, P46=0.49740423017416585, P47=0.5396329761095794, P48=0.5801325336137748, P49=0.602578820078212, P50=0.6717678377729364, P51=0.699893099922733, P52=0.00017402612390153205, P53=0.0001923857353311894, P54=0.00019943208326006143, P55=0.0001456743304458775, P56=8.02775079767189e-05, P57=2.9547842410737494e-05, P58=2.4209496544150538e-05, P59=2.542959685843604e-05, P60=3.5632363123004586e-05, P61=4.772497794655996e-05, P62=5.1146609284478726e-05, P63=4.865969758533967e-05, P64=5.498957067089868e-05, P65=0.0001089759972719352, P66=0.00015864817941734985, P67=0.00015858919319588534, P68=0.00022108812328112056, P69=0.0009645197774522994, P70=0.010989984093923476, P71=0.09325028546670484, P72=0.16442160321554955, P73=0.18947788962989784, P74=0.35311479884997693, P75=0.5424570911140257, P76=0.6338322097308376, P77=1.1843357363899185, P78=1.658634734626484, P79=1.2243623453307868, P80=0.7773751796206649, P81=0.6997400312595891, P82=0.8069915979032849, P83=1.1575819119486688, P84=1.4757357546505687, P85=3.0417483927921167, P86=3.9426617693972754, P87=5.3607522265455225, P88=6.153695575752626, P89=4.191405598152639, P90=2.8884101479069346, P91=0.6665066261446755, P92=0.42980975980044667, P93=0.7814031745432177, P94=0.7830979991279642, P95=0.6758993966955977, P96=0.7170487741887098, P97=1.9987185457215155, P98=1.4684577086341668, P99=0.517602762379373, P100=0.5042610244364849, P101=0.37180482279965654, P102=0.599601390948188, P103=0.28192299077657434, P104=0.244629896238034, P105=0.5709846419094686, P106=0.39449257437301194, P107=0.2682420712509984, P108=1.0336635668223804, P109=1.3732262235007244, P110=0.759119640183432, P111=0.3757158591327414, P112=0.28385601670465704, P113=0.5238532813911061, P114=1.4343740106061877, P115=0.8806739643273147, P116=0.06361953436949289, P117=0.11246588513234074, P118=0.4045360528017837, P119=0.5065970702863237, P120=0.3478073411175537, P121=0.10530964067381664, P122=0.02834098623690034, P123=0.20302300258111114, P124=0.13913414363892482, P125=0.3293014869018453, P126=0.35336784208571237, P127=0.05960821684678594, P128=0.05277470896905354, P129=0.02937342698562712, P130=0.09770209206994579, P131=0.09707123320353835, P132=0.18083235386401722, P133=0.1367746644386338, P134=0.0318237429678317, P135=0.014494506537454021, P136=0.01615626743673024, P137=0.04540434097363256, P138=0.046010952495268466, P139=0.027768907307859515, P140=0.01799996840430685, P141=0.05243328959064912, P142=0.06734965594024997, P143=0.021499722826954462, P144=0.04409163473341585, P145=0.08766557687369884, P146=0.026982489738254418, P147=0.02121085360923935, P148=0.06301849105529139, P149=0.05422252361958023, P150=0.006408402041364364, P151=0.002526694853769912, P152=0.0009999522346064806, P153=0.0004136773924034535, P154=0.00040561766801377563, P155=0.0005554207647091025, P156=0.0008798651870962866, P157=0.0013555402505516139, P158=0.0016061753313336142, P159=0.0013717623025672223, P160=0.0014481404163130445, P161=0.0013560189708642883, P162=0.002002554184001545, P163=0.002445701506351089, P164=0.0034733277803389966, P165=0.005922873850612082, P166=0.0062983769084915256, P167=0.006456614678455679, P168=0.0075895761440001215, P169=0.004104533549847253, P170=0.0022408633871710596, P171=0.0024309625007178366, P172=0.003618844090841528, P173=0.0035792939920004864, P174=0.0033481648669510225, P175=0.004110529408450292, P176=0.004470127829846097, P177=0.00350876857155974, P178=0.001425810090738916, P179=0.0001666966196063995),Row( P0=-645.0334117053701, P1=41.37454381249808, P2=-13.152287421048477, P3=-8.792414049152661, P4=-23.152376324768184, P5=-13.140405865194216, P6=-5.7661268130349255, P7=-5.105357587422156, P8=-11.229008397405638, P9=-7.413126962289513, P10=-6.92487193797809, P11=-6.2700835404927, P12=-5.323274655512079, P13=-2.407348520396702, P14=-2.222682811936562, P15=-1.2705895032569112, P16=-1.620522019545148, P17=-0.00269551953089992, P18=3.349432290079044, P19=0.8818129239455598, P20=6.28641403883301, P21=3.677933099296093, P22=-0.01314887164217339, P23=3.975602040757673, P24=-1.171105908647189, P25=0.6738208915546576, P26=-2.132214953514142, P27=-3.649169500850327, P28=-3.24588512065002, P29=-0.37164874889982963, P30=-0.8490201664356483, P31=1.0243701140434411, P32=-2.0394995208125724, P33=-0.9652106160603107, P34=-1.5280918815986042, P35=1.3703330054937102, P36=0.6412564683483973, P37=-0.06238412600007886, P38=-2.172905547078871, P39=-1.2830781846493355, P40=0.49711767169112364, P41=0.5580147568466314, P42=0.5504922670488975, P43=0.5108914970086842, P44=0.5287565653030314, P45=0.5350535653320131, P46=0.5533604576592368, P47=0.5595198311145583, P48=0.5087590695745318, P49=0.4819727233408699, P50=0.5105314555990201, P51=0.4974342027052935, P52=2.026328714512219e-06, P53=1.4513588906448618e-06, P54=1.631190307283739e-06, P55=4.856348504108311e-06, P56=1.3837340624779833e-05, P57=2.6610059192936872e-05, P58=2.0420197171427945e-05, P59=2.8702143354955586e-05, P60=0.00020612512772223503, P61=0.0015248312288032753, P62=0.014645966619589276, P63=0.030911617952073946, P64=0.024158594856252255, P65=0.045881780740637104, P66=0.08237311050724301, P67=0.06174330646087372, P68=0.01780044889026575, P69=0.014357525241552918, P70=0.007383598867989396, P71=0.005621932369281615, P72=0.01818715916255557, P73=0.015711754156191457, P74=0.01730297830494306, P75=0.03609946826199149, P76=0.03164183180803425, P77=0.024343545708885122, P78=0.01960921082497191, P79=0.02879675112404551, P80=0.11273474614202537, P81=0.1487111922177906, P82=0.05246459999442237, P83=0.02185898687708759, P84=0.008517260666344638, P85=0.01392816390103279, P86=0.030800989532641047, P87=0.03077958536764627, P88=0.018124316417377984, P89=0.00962886229951371, P90=0.007025865315605924, P91=0.010298930010438075, P92=0.019891801155531297, P93=0.01870226624766578, P94=0.015912547827107736, P95=0.0400393364055889, P96=0.03049098839279761, P97=0.0242149861414713, P98=0.013253998618848075, P99=0.0037203638792143116, P100=0.0029992455428837434, P101=0.005128474816338007, P102=0.007746711476459881, P103=0.0052336812252096986, P104=0.003774212666233369, P105=0.0009614097081232058, P106=0.001062736960491439, P107=0.0006531151267556081, P108=0.0006044296023187284, P109=0.0008998362443409712, P110=0.0008439372633491952, P111=0.0004888330425881366, P112=0.000318581598040924, P113=0.0004264502323072739, P114=0.0008124680234634302, P115=0.0006544468445407849, P116=0.0002587344959123215, P117=0.00016968046115245375, P118=0.00032671756228850604, P119=0.00020450381162637413, P120=0.00015154891608469432, P121=0.00033986146741146576, P122=0.0001994271612966632, P123=0.00016739603688449914, P124=0.0004947923907883457, P125=0.0002708990750948404, P126=0.0009173999578739145, P127=0.00123647131039056, P128=0.00016755066319977917, P129=0.0011173900623942838, P130=0.004372172823896725, P131=0.0021794813600769084, P132=0.0002679796689833565, P133=0.0009421043108360078, P134=0.0005964715415377202, P135=0.0005486503966458368, P136=0.0006005929237566244, P137=0.00015842887947499444, P138=0.0008159621656564551, P139=0.0004387016750991914, P140=0.00034932326100470463, P141=0.0003148422529036113, P142=0.0006095365304501338, P143=0.0006641774300233573, P144=7.702052744855172e-05, P145=7.277855066995617e-05, P146=3.736020572350107e-05, P147=7.105881728579478e-05, P148=0.00011171735972239014, P149=0.00018198347014751704, P150=0.00026535905950934695, P151=0.0005715207570279281, P152=0.0008332317514643525, P153=0.0010638510455026544, P154=0.00035795464066198186, P155=0.00015646684573940938, P156=7.381614219917252e-05, P157=0.00010677217687666906, P158=0.00027335613161338516, P159=0.0003951838068705204, P160=0.0011787081868688393, P161=0.0010269259474893868, P162=0.0009433677693497603, P163=0.0008570417728760658, P164=0.0007178982300726484, P165=0.0011474977348616515, P166=0.001025252038397606, P167=0.0007117647616179674, P168=0.00043488148056647645, P169=0.0004939355697408434, P170=0.0008960681012383195, P171=0.0010008734878731206, P172=0.0009699369007939875, P173=0.0006996295719027275, P174=0.00028369625075829304, P175=0.000251398697768326, P176=0.00038388062684577046, P177=0.000471366635957687, P178=0.0002649907069224029, P179=3.1825063358912146e-05),Row( P0=-665.8131993391977, P1=54.950725064525955, P2=-7.224557420867745, P3=-5.653881718147779, P4=-17.604924781266114, P5=-13.959967251228349, P6=-2.1515554212060013, P7=-5.832200999270335, P8=-10.346770839963764, P9=-6.512207554263045, P10=-7.071009895566962, P11=-6.90689239965119, P12=-9.929596626502597, P13=-1.7051360673763534, P14=-2.22369043562289, P15=-0.4722414170289621, P16=0.5774841312180021, P17=0.20444209819292364, P18=2.9152136774029054, P19=0.013401841384673397, P20=1.4105950347057032, P21=2.4362510258396144, P22=0.48917333038532623, P23=2.96029140588248, P24=-1.496530035442193, P25=-0.8877776036204221, P26=-2.5593301032242093, P27=-0.32565428555268294, P28=-1.818745891425168, P29=0.2638875371117858, P30=-1.8691026120361194, P31=0.8375129165619734, P32=-0.47786761198926075, P33=-0.4500642307821959, P34=-2.3882901450973657, P35=-0.45849613225368574, P36=-0.740219085416572, P37=0.45335768565261625, P38=-2.9948571527659467, P39=-1.9450949817297383, P40=0.44754794546731147, P41=0.5166377192298722, P42=0.5103111342351476, P43=0.4547496805587849, P44=0.4748397559110303, P45=0.46267804649015337, P46=0.45369170364145317, P47=0.4486470150388571, P48=0.4427336224240572, P49=0.4245760520087334, P50=0.48633907926927494, P51=0.4727440510579113, P52=1.0075257403513472e-06, P53=1.8576981395853366e-06, P54=4.542483291295741e-06, P55=1.6309006699565794e-05, P56=3.089305807719598e-05, P57=3.50657801173455e-05, P58=1.9132413196474407e-05, P59=2.6411177242988676e-05, P60=0.0010695644392506936, P61=0.007457176202434219, P62=0.012497030524499585, P63=0.012949769961840978, P64=0.0061966370579347625, P65=0.02190534902705959, P66=0.053165884688706616, P67=0.02945413766566304, P68=0.01445669760731012, P69=0.01713903845738509, P70=0.009515571531826725, P71=0.002628635714034385, P72=0.0017903493541742545, P73=0.0013496757455955042, P74=0.006465155535809054, P75=0.010631324257786717, P76=0.012474705168308198, P77=0.008326208813663593, P78=0.0021504571254654958, P79=0.0013443955264704859, P80=0.0036154997614135606, P81=0.00407859418710439, P82=0.003505984171072886, P83=0.0036496919491397525, P84=0.02455181812370418, P85=0.05730173145369082, P86=0.03751189509523327, P87=0.048697000648815894, P88=0.04403727930716788, P89=0.015831162961284093, P90=0.0060874021115712275, P91=0.005670560597483414, P92=0.0033733690193363908, P93=0.003786999451905764, P94=0.005977967341512654, P95=0.007356704860328568, P96=0.0019925064388837174, P97=0.001656852278747589, P98=0.0010976140983247772, P99=0.00036402349841321635, P100=0.002825473291226108, P101=0.0124746429234288, P102=0.008561697790134314, P103=0.002423069036111258, P104=0.0004983772649248778, P105=0.0003164136559084264, P106=0.0036002363109902233, P107=0.002505140632737573, P108=0.003389041989706274, P109=0.001597559523511462, P110=0.0006005433760541581, P111=0.00037755691857439746, P112=0.00013218505471332065, P113=0.0004523057995438204, P114=0.00147948130850225, P115=0.0009211560326965, P116=0.00014732938067846602, P117=3.031273446154544e-05, P118=0.00022451372879389627, P119=0.0003616793722637839, P120=5.2075785417813434e-05, P121=3.132517676152162e-05, P122=9.31218370717175e-05, P123=0.00013214296270880988, P124=6.136083917924594e-05, P125=0.00019586144198394768, P126=0.0002909506279023297, P127=0.00017712640298644174, P128=0.0002026300142396646, P129=0.00024688177549230753, P130=0.0001404491694813119, P131=9.026519877269586e-05, P132=7.307989415664769e-05, P133=7.846054736857752e-05, P134=6.378654832478697e-05, P135=5.505805123267386e-05, P136=3.130658933420434e-05, P137=3.237601414370513e-05, P138=4.196831457692275e-05, P139=5.600169121370258e-05, P140=6.0286550856257654e-05, P141=4.2883084894491016e-05, P142=3.716622682034223e-05, P143=2.1925884293074734e-05, P144=1.5155060890382991e-05, P145=1.260920227264829e-05, P146=1.2909497146576806e-05, P147=2.168724060716703e-05, P148=5.196233853101277e-05, P149=3.961799451405456e-05, P150=2.3606203537367893e-05, P151=2.597880968146314e-05, P152=3.0950974586085337e-05, P153=1.6153730298170224e-05, P154=1.5204319356624828e-05, P155=1.5730797493462998e-05, P156=1.3982024832955353e-05, P157=1.3735716707635512e-05, P158=1.1619680987806035e-05, P159=1.6149673428242336e-05, P160=2.492631421588554e-05, P161=3.617081108536098e-05, P162=3.6199936912768115e-05, P163=9.926957254037473e-05, P164=0.00013682038934851627, P165=0.00012610243913852384, P166=9.160781356284529e-05, P167=4.722677420974562e-05, P168=4.240512721681541e-05, P169=3.695228564313392e-05, P170=5.5434128681474925e-05, P171=6.689029174791109e-05, P172=8.088030204833072e-05, P173=9.945482217823508e-05, P174=5.412530724728796e-05, P175=5.816399676939473e-05, P176=0.00010421674118517386, P177=0.00011940776342098851, P178=4.074328812211412e-05, P179=3.91956514192794e-06),Row( P0=-682.5788042972864, P1=49.570405442187194, P2=-12.898659904832826, P3=-6.776260265229309, P4=-17.22826935343048, P5=-11.198831507909603, P6=-5.7350302019250154, P7=-3.548379780490165, P8=-9.73787696588666, P9=-7.714768047826488, P10=-4.528407403755291, P11=-5.328452457737263, P12=-9.007356018332935, P13=-2.0636252340003542, P14=-2.515946328671189, P15=-1.0340266859634641, P16=0.08740097901986864, P17=-0.03206110853496122, P18=2.840894945288064, P19=1.958754230362742, P20=3.620769684599293, P21=6.6374140720431285, P22=3.4516468831435865, P23=4.68791265603196, P24=0.42954856908004685, P25=1.2820673057770189, P26=-1.7333697579824954, P27=-2.6319301031106224, P28=-1.6015293163401978, P29=0.01860192242008847, P30=0.31607840298866774, P31=-0.9681934243545859, P32=-1.6470072814925438, P33=-1.1367293686991018, P34=-0.8052905556191633, P35=-1.1423782728044036, P36=-2.8680078461768654, P37=-0.3185637344397323, P38=0.020808665951820043, P39=1.6348022136117213, P40=0.5897518976494467, P41=0.5780085062693339, P42=0.5723358320444383, P43=0.6041428590784832, P44=0.6131591064752429, P45=0.5906847594582015, P46=0.6057761721473784, P47=0.5729964983687246, P48=0.5607121510308414, P49=0.5482017123450195, P50=0.5512031932506556, P51=0.5738027815061362, P52=1.2679708740241652e-06, P53=2.1624258640488133e-06, P54=2.012510450499252e-06, P55=6.0370586872982224e-06, P56=1.558992407090673e-05, P57=2.2227023847596965e-05, P58=9.020486709941837e-06, P59=6.268682083069997e-06, P60=0.00010069397648392846, P61=0.002801280698846562, P62=0.015719655174010663, P63=0.016452443120517395, P64=0.007160566110512773, P65=0.009698829691589202, P66=0.02765357431035803, P67=0.038769475948283505, P68=0.013959212215798843, P69=0.0033489154184934674, P70=0.002438040450728343, P71=0.002505364372238413, P72=0.0018789884371266004, P73=0.002748533298387057, P74=0.007877865545503194, P75=0.011238571056038404, P76=0.008354474240842832, P77=0.01132687973965513, P78=0.012732041987500727, P79=0.009310298812332715, P80=0.00640359094097173, P81=0.004311611838631796, P82=0.01755474942324067, P83=0.02557846908386981, P84=0.026124048882995757, P85=0.02447392657037888, P86=0.009397637628848713, P87=0.00559994759133712, P88=0.00669537136037536, P89=0.009738834023177068, P90=0.011295923435975231, P91=0.0058441197553948925, P92=0.001662728299496547, P93=0.0029486992245033716, P94=0.0029370383077253594, P95=0.0041900529949053305, P96=0.002209237249870164, P97=0.002764665674227183, P98=0.002994180824672339, P99=0.0009243149220263686, P100=0.0007810166491002503, P101=0.0018424743225166553, P102=0.012913751279012665, P103=0.014618881244195723, P104=0.0014092953327656775, P105=0.0004141729973319088, P106=0.00044307947285349847, P107=0.0001709831139990786, P108=0.00014247741321882888, P109=0.00013412571829632183, P110=0.0027020921240109696, P111=0.02893842001343299, P112=0.03244276839326856, P113=0.0006575867294028554, P114=6.589012867057746e-05, P115=4.9051119397013125e-05, P116=9.214297053456451e-05, P117=0.0001602663145170121, P118=0.0011217292095195204, P119=0.0011725528371779972, P120=6.789688477579794e-05, P121=0.00013917833238211906, P122=0.00024706515824533267, P123=0.0005083283751193497, P124=0.0004353163238302785, P125=8.439784761270074e-05, P126=0.00018720806117393992, P127=0.0004291124231084127, P128=9.904012538317989e-05, P129=0.00010734187777075658, P130=0.0001386225403082782, P131=9.058239241003626e-05, P132=0.00010620867443282088, P133=4.008145397407533e-05, P134=6.592323587602539e-05, P135=6.96022244443084e-05, P136=2.769808602307344e-05, P137=2.489149601948476e-05, P138=2.5865143013581464e-05, P139=2.9602463271232078e-05, P140=6.525534603707779e-05, P141=2.495505638004883e-05, P142=3.404151645663913e-05, P143=2.5339069225234627e-05, P144=2.161010802300323e-05, P145=2.5919127478108795e-05, P146=2.3769911507810114e-05, P147=4.0091344025138136e-05, P148=5.5529675660365544e-05, P149=0.00011255126808689376, P150=0.00013565431230690977, P151=7.219872273461416e-05, P152=0.00013753421628107064, P153=6.491067997834266e-05, P154=2.896321534042218e-05, P155=2.772023636111038e-05, P156=2.1121639382560416e-05, P157=1.9116186072767733e-05, P158=2.5254899930674075e-05, P159=3.2898236793144055e-05, P160=3.4405897570124596e-05, P161=5.676240633152001e-05, P162=7.52790438522868e-05, P163=0.00010323310229101286, P164=0.00014160255324998248, P165=5.197830505271387e-05, P166=4.0642432476915704e-05, P167=2.2251282932154726e-05, P168=2.8372370458337555e-05, P169=2.7320111885513356e-05, P170=4.771962051663183e-05, P171=5.0407944935173166e-05, P172=0.00011798534775269524, P173=4.0458429623244394e-05, P174=1.8284399516584132e-05, P175=2.145924390132844e-05, P176=3.855908881440173e-05, P177=5.124944819097077e-05, P178=2.8734590885549975e-05, P179=3.5155400019748343e-06),Row( P0=-698.373283969555, P1=46.91303877060125, P2=-9.63563849571654, P3=-10.609125060068353, P4=-19.14173710013786, P5=-15.001302518691435, P6=-5.407554909373337, P7=-4.436699294291263, P8=-11.864698488202635, P9=-7.681680406352022, P10=-8.249593863218454, P11=-3.440901549720337, P12=-12.227222180221673, P13=-4.219189890521147, P14=-4.160790371571478, P15=-6.331453329040409, P16=0.5343893512713287, P17=0.38289441164726845, P18=4.048330399337975, P19=5.365176958337315, P20=3.7656570080640996, P21=5.162957614692821, P22=0.5488439120444579, P23=3.261826003239306, P24=0.9544240376243982, P25=0.9107619887814871, P26=0.7493156757136727, P27=1.2011233993711796, P28=2.700257228100176, P29=0.9280330629416743, P30=-2.1063079184527727, P31=-0.2858485022424933, P32=0.5875531180910291, P33=1.1928582076124752, P34=-1.1555165018874003, P35=0.36518020559254805, P36=-3.061780225471942, P37=-0.5465312839099332, P38=-1.6457778798712732, P39=1.6432058744959395, P40=0.4919537231009494, P41=0.4723498614015596, P42=0.4747800119360631, P43=0.5081972234983294, P44=0.48518116237654463, P45=0.4525502804360969, P46=0.44729215627182906, P47=0.45715899865931764, P48=0.48451721157849836, P49=0.4703568442784936, P50=0.4648080155687697, P51=0.47666479492698055, P52=9.409077976195932e-07, P53=9.65232681105003e-07, P54=4.186520092682016e-07, P55=8.808744539288056e-07, P56=3.020606341557698e-06, P57=4.424710545885896e-06, P58=2.5634956193403463e-06, P59=5.921294870489371e-05, P60=0.0019602353106952576, P61=0.0057061130917520015, P62=0.005670358780338088, P63=0.006657190721859766, P64=0.012088123688039585, P65=0.010407734881306056, P66=0.017376774837231816, P67=0.015929309483918307, P68=0.0038844073073606216, P69=0.0008568008621884386, P70=0.0010361994868116936, P71=0.0011325840931698254, P72=0.0010343934819050447, P73=0.0017891421674679601, P74=0.0015685528644893807, P75=0.002824605517771877, P76=0.015483691904289443, P77=0.061185137025565293, P78=0.04046724389316008, P79=0.01730216740144668, P80=0.0071204618272015414, P81=0.004978052584826683, P82=0.004418784271213235, P83=0.002389982432689936, P84=0.005277794665272342, P85=0.008572106817841483, P86=0.0021500115293637064, P87=0.0008325182123576378, P88=0.003493855860639882, P89=0.010921275860761517, P90=0.015036786379752028, P91=0.0132795293264459, P92=0.008821922681964897, P93=0.009011879227170215, P94=0.03172562764584309, P95=0.025326254314268694, P96=0.0019451218242968868, P97=0.00039540490970180875, P98=0.0006827991695509463, P99=0.0043359607238412125, P100=0.007607693084274484, P101=0.005102313045022169, P102=0.0037872242866275477, P103=0.004088380307630067, P104=0.0036833244376998186, P105=0.0012508493698565085, P106=0.0005030040591869424, P107=0.0003171504656585816, P108=0.00022789162622620683, P109=0.00020215418676679147, P110=0.0008179357898679302, P111=0.00037923346672007576, P112=0.0001003097650240549, P113=0.0003369455754795034, P114=0.00038422468112760035, P115=0.0006668358589920264, P116=0.0012022910846948391, P117=0.0003211006018232814, P118=3.8092607153038776e-05, P119=1.911819737330367e-05, P120=0.00012135340061307531, P121=0.0001852593223122734, P122=2.4601719723360713e-05, P123=7.618961076904546e-05, P124=0.00020360737199218824, P125=5.338065570900331e-05, P126=0.00011459745010949688, P127=0.00022781408576749934, P128=7.365644269846566e-05, P129=0.00019802745483649925, P130=0.00037067606007249495, P131=0.0002526915404606576, P132=0.0009925625831265856, P133=0.0006763783263565199, P134=3.217785892046158e-05, P135=7.76844945385384e-05, P136=2.0091909931831585e-05, P137=4.30197012709859e-05, P138=2.4695753583355978e-05, P139=3.333449746936913e-05, P140=2.951121320022134e-05, P141=0.00010826344240093852, P142=7.846087467238533e-05, P143=3.716364974665768e-05, P144=1.8033741794763766e-05, P145=1.565230241594697e-05, P146=1.5814337969770644e-05, P147=2.5364479752047764e-05, P148=4.8639816998978534e-05, P149=3.853619424700132e-05, P150=4.288042527216462e-05, P151=6.307073651156703e-05, P152=4.070596925446351e-05, P153=2.0433479785930436e-05, P154=1.6589608245456267e-05, P155=2.5909434723140758e-05, P156=2.232256922235918e-05, P157=1.2474307137226037e-05, P158=2.1251872778479304e-05, P159=4.255776058224459e-05, P160=0.00014626169100471614, P161=0.00013064181446629386, P162=0.00012467390952081983, P163=0.000227544969319988, P164=0.00017949506355762818, P165=0.00010811093098966406, P166=9.770664056285473e-05, P167=5.408775848966511e-05, P168=9.195808349179861e-05, P169=0.00011129242104649436, P170=0.0002017870916832598, P171=0.00031342572640464096, P172=0.0002288654409839309, P173=0.00017093010137103008, P174=9.031868084305153e-05, P175=7.037769403244807e-05, P176=7.74684012942676e-05, P177=8.948545665304746e-05, P178=9.022081686517038e-05, P179=9.142901872575762e-06),Row( P0=-610.5520415886538, P1=47.79577365903829, P2=-13.34829502860157, P3=-10.138704318336991, P4=-22.391341705554634, P5=-13.223539456830553, P6=-8.95739602721782, P7=-3.5995587237071507, P8=-7.086873855635345, P9=-12.746667084102613, P10=-8.733296414985547, P11=-0.8107854426871064, P12=-12.29364280084996, P13=-6.725419338764615, P14=-4.43103346956861, P15=-7.970404083477468, P16=-2.307228623219737, P17=1.040428206320493, P18=2.4385386913023304, P19=4.887476844298513, P20=6.333076753826093, P21=4.977831418153195, P22=2.6472193471705108, P23=1.6654317555231815, P24=-2.115259000062147, P25=-2.3465902974174697, P26=-1.9504549728910086, P27=-2.625970873706909, P28=-2.017053145330214, P29=1.363723972349851, P30=-3.1988668110113676, P31=0.7095477229395101, P32=-1.627176798027891, P33=-1.206580308540134, P34=-1.031190306642765, P35=0.24646356006223946, P36=-1.2919539412905627, P37=-0.5535869254369536, P38=-1.0171877288202027, P39=0.9609406760503115, P40=0.6158078596055736, P41=0.6087165858326145, P42=0.5785122924904919, P43=0.5974573238512532, P44=0.603754004128332, P45=0.58684386283727, P46=0.6562466584159204, P47=0.6782695978488047, P48=0.7259638918160233, P49=0.7328089459719953, P50=0.7255517018120262, P51=0.6653471588468393, P52=2.9006227748814475e-06, P53=4.275891148804627e-06, P54=7.720789788665264e-06, P55=1.875020431366076e-05, P56=3.847338320185036e-05, P57=0.00010269379088107023, P58=0.0001134073236466184, P59=9.881424240165344e-05, P60=0.00040125213808604417, P61=0.0027751252129539186, P62=0.014607598237653728, P63=0.03782646679850058, P64=0.04746503963804636, P65=0.052171340612230076, P66=0.0750440556836626, P67=0.0540999372665952, P68=0.012335841164491174, P69=0.0017011776988371165, P70=0.0007482895514154627, P71=0.0006193653266225275, P72=0.001024495634798798, P73=0.001137631282584752, P74=0.006696682833846712, P75=0.01643913530540175, P76=0.0333806123787738, P77=0.09054388187875369, P78=0.09500690995473833, P79=0.06266426948719393, P80=0.17433094401958585, P81=0.17057654074874337, P82=0.07534100408391273, P83=0.03788743425082341, P84=0.011663946810959802, P85=0.0054271489614325106, P86=0.003975819487852493, P87=0.005143188211773474, P88=0.011273873835647404, P89=0.014044421084510525, P90=0.01617542639704417, P91=0.012088091434622625, P92=0.005384155010029954, P93=0.005378868763234984, P94=0.00691140394936127, P95=0.008794533247654076, P96=0.004088247344223217, P97=0.006608988287065845, P98=0.010570401603835088, P99=0.005369982717694718, P100=0.003700014629945548, P101=0.0021223579218901914, P102=0.0036064714652133153, P103=0.005334009182133729, P104=0.003365421411935616, P105=0.0011713462679381816, P106=0.0006977495891732566, P107=0.0002993126037342163, P108=0.0002410411263211684, P109=0.00038750644511128125, P110=0.00120202517307749, P111=0.001111634588167014, P112=0.00047698206499406827, P113=0.0002874506842517886, P114=0.00046103555484970133, P115=0.0016051528168643603, P116=0.0025814842203634157, P117=0.0016394934219364204, P118=0.0007516322155125018, P119=0.0009987679175791346, P120=0.0007542008909477773, P121=0.0010055335435033138, P122=0.0008026207522386006, P123=0.0006888426332981829, P124=0.0004971298650703191, P125=0.0006498187655448355, P126=0.0005203797212086944, P127=0.00034659520617602856, P128=0.00022291473389009906, P129=0.0005717542691291962, P130=0.0009603053146852949, P131=0.00044134772620759555, P132=0.0013333558480978565, P133=0.0009643354451956049, P134=0.0002406760261164956, P135=0.00022170881047990006, P136=8.055073591031057e-05, P137=6.613227036883817e-05, P138=9.837012665269684e-05, P139=0.00014088773971493803, P140=0.00025265203170333437, P141=0.00031546119525814863, P142=0.0004862544374343888, P143=0.00047474228841299927, P144=0.00037329443806037414, P145=0.00033329163516327787, P146=0.0002675580589993523, P147=0.0003159401574710375, P148=0.00033912927711134724, P149=0.0005182773429779466, P150=0.00045088224094276216, P151=0.0002873197948091171, P152=0.0001805492090009783, P153=0.00012621802782299948, P154=0.00012780478409050845, P155=0.0001422751067523903, P156=0.00019807902342425192, P157=0.00015567857777418027, P158=0.00010802958721860986, P159=7.540836427662438e-05, P160=8.041813638555078e-05, P161=9.899781779030332e-05, P162=0.00011283170815421988, P163=0.00010532324302022706, P164=0.00016942626984624423, P165=0.00019621865978440112, P166=0.00019655887749455526, P167=0.00019328776799365502, P168=0.00011087325631128363, P169=4.932542382317997e-05, P170=4.1249689707110615e-05, P171=8.144574944334528e-05, P172=9.624862320405411e-05, P173=0.00010522475886250248, P174=5.901135721638246e-05, P175=6.24986491687591e-05, P176=6.191707473691868e-05, P177=9.676548253636682e-05, P178=8.405906908638366e-05, P179=6.510015637706768e-06),Row( P0=-645.8611888542824, P1=48.571118177823976, P2=-12.50164986738741, P3=-4.6836110917825025, P4=-17.894958741600558, P5=-7.955087705834675, P6=-9.757272619293085, P7=-1.4273783078000686, P8=-5.130734771238697, P9=-9.687515155683526, P10=-9.31253181523079, P11=-1.6428176459535455, P12=-7.472577813946267, P13=-2.535840496910845, P14=1.1578327622548348, P15=-5.074495339110018, P16=-1.0279185515690372, P17=-0.6068498626231602, P18=-2.6200867913262407, P19=2.0770884636637943, P20=1.3601528466295774, P21=4.103358429492883, P22=1.7582303534728423, P23=1.1020787409220767, P24=-1.496253056429177, P25=-3.030742418269416, P26=-3.098289005237535, P27=-1.8770367157494359, P28=-0.7289725909846154, P29=-0.32614294275740807, P30=-2.7486737051220085, P31=-0.06150082670285559, P32=-1.8759420402864595, P33=0.07542221061526307, P34=-2.730691288007225, P35=-2.46804196900582, P36=-1.42368371519572, P37=0.18048092248100447, P38=-1.7579740446360108, P39=0.41733186248152054, P40=0.5421706192532205, P41=0.5831933680153373, P42=0.5428738688503377, P43=0.4905758964976298, P44=0.49369608687703537, P45=0.4778664066778321, P46=0.506051053324183, P47=0.5050227872767402, P48=0.5168497130920946, P49=0.5374044599205773, P50=0.4940375438629679, P51=0.49110349166650397, P52=6.220395048067694e-06, P53=9.972595568682401e-06, P54=2.6756335191064567e-05, P55=7.137076779996823e-05, P56=0.00014167617413328044, P57=0.00022347921522742386, P58=0.00019778178620385122, P59=0.00020635108777765357, P60=0.0012136822084587524, P61=0.005203900445980436, P62=0.018018414540325337, P63=0.026425607576846952, P64=0.032810761214177996, P65=0.06434509273760088, P66=0.02481127149870257, P67=0.013762201064900123, P68=0.02215563680194508, P69=0.036321480234092034, P70=0.023976439406469045, P71=0.006583367518593237, P72=0.0037134958337405473, P73=0.005248276658127553, P74=0.022556158198786445, P75=0.02408060831875162, P76=0.014125624833413751, P77=0.024237718337347308, P78=0.044694868533624275, P79=0.0588960012167843, P80=0.06641934846846712, P81=0.01939969744291951, P82=0.005241736525805804, P83=0.007198000474540686, P84=0.015618344219703035, P85=0.04183248852297328, P86=0.0592803028565179, P87=0.04562149029677336, P88=0.028885406570345464, P89=0.01985501462696522, P90=0.020856860124907093, P91=0.03107229278994509, P92=0.03848223641033513, P93=0.018051210170857035, P94=0.005492146194456585, P95=0.0071092091567785285, P96=0.015329878879151429, P97=0.011145264962363444, P98=0.004264412463628368, P99=0.0027211789184811907, P100=0.0019707299708787668, P101=0.0025320837262503668, P102=0.004944457338135924, P103=0.0014447201777109944, P104=0.0006067926835446467, P105=0.0008527497726004873, P106=0.0011237561618179254, P107=0.001308909376571396, P108=0.003238020719369452, P109=0.002064084803102415, P110=0.0010929426603021038, P111=0.0012233578865537993, P112=0.0007990176730973573, P113=0.0011186385381932375, P114=0.0012606826413280089, P115=0.0009748942328232394, P116=0.000497415282222415, P117=0.0005172810478469365, P118=0.0010359319686911589, P119=0.0005008929807632795, P120=0.00041642724689276565, P121=0.0008711240846693869, P122=0.000899936725900189, P123=0.0005023050097735822, P124=0.0002872188030085754, P125=0.0002483166915922158, P126=0.0004249445851349712, P127=0.0005532475840035151, P128=0.00028450983801656343, P129=0.00023666724208962203, P130=0.0002869437786727496, P131=0.00022374356544434365, P132=0.00011743432772157849, P133=0.00011968985704359188, P134=9.285821713231613e-05, P135=4.902293079665246e-05, P136=7.590278038018607e-05, P137=7.02570962067458e-05, P138=5.560712618993994e-05, P139=9.780533322024875e-05, P140=8.506099821523291e-05, P141=0.00017440303823118146, P142=0.0002992488855186305, P143=0.0002825500545439329, P144=0.00029797232920174895, P145=0.0002782877972580177, P146=0.00031656735696437914, P147=0.0004095632917987307, P148=0.00037162629988513616, P149=0.0005436395251744897, P150=0.00048497146850395455, P151=0.0003309279866442958, P152=0.00021612052906033895, P153=0.00013555814855916766, P154=0.0001257282714966404, P155=0.00012664231785789351, P156=0.00023488982252164997, P157=0.00017859230401932755, P158=0.0002749634931578643, P159=0.0004874559744870906, P160=0.0005782184384407072, P161=0.0004056662685195025, P162=0.0003043894633202897, P163=0.0003522132126091021, P164=0.00024708887669230996, P165=0.00016743287930257034, P166=0.00027682761779277164, P167=0.00035069980023813433, P168=0.00015246790041857872, P169=0.00019513368275959417, P170=0.00011745938815876999, P171=9.145100877348036e-05, P172=6.55904662568535e-05, P173=0.00016240505954523082, P174=0.00015256814574185143, P175=0.00026942485238044, P176=0.0001577913090956655, P177=0.000124474133421646, P178=6.474480029940982e-05, P179=8.886889934909601e-06),Row( P0=-687.2775346463877, P1=48.00996038516823, P2=-10.94722703217801, P3=-11.554346627894558, P4=-17.392223399493766, P5=-16.18677951614446, P6=-4.341066192307853, P7=-2.398294294559447, P8=-11.523759214002888, P9=-6.780255304279279, P10=-9.088503096587516, P11=-4.966870860492391, P12=-11.450514870121246, P13=-5.032738158427426, P14=-6.715281583469097, P15=-7.8497239149543265, P16=-3.179447354828887, P17=-4.615481090898263, P18=-0.4173062635609912, P19=2.4810492178469827, P20=4.003005833075506, P21=6.9197274573851635, P22=4.3697419860070745, P23=5.670795692270112, P24=2.365307477197679, P25=4.601679432595184, P26=3.874405953462355, P27=4.414784794604391, P28=3.3351818414112784, P29=1.0047512005629042, P30=-2.8328466709830415, P31=-1.0608224414717953, P32=1.3377394413281107, P33=1.0400634373112054, P34=-1.42574333631705, P35=-3.140637306166469, P36=-3.4697369858501528, P37=-0.16363745745142558, P38=0.8641444919892646, P39=2.230636335915853, P40=0.5436178969792329, P41=0.5634299242607764, P42=0.5644412465445784, P43=0.6282493362432116, P44=0.654338592338924, P45=0.5568394697657453, P46=0.5059822029923967, P47=0.4919102244314177, P48=0.5736728494061929, P49=0.5248926468121585, P50=0.495789813636572, P51=0.5046740345753806, P52=1.0677439915523057e-06, P53=1.4090050883998402e-06, P54=6.950460988843498e-07, P55=7.46245690112981e-07, P56=1.9902531493456005e-06, P57=6.555754264985406e-06, P58=5.002663960093796e-06, P59=5.780433919801892e-05, P60=0.0022918405430784063, P61=0.013836233322641146, P62=0.021985988385816307, P63=0.011009433413734865, P64=0.010019158975533606, P65=0.008152142452064196, P66=0.0013042769752774882, P67=0.00040988430746299, P68=0.00021378916561798898, P69=0.0003022317732182259, P70=0.0006760207369165968, P71=0.002459193169348856, P72=0.00848009199985353, P73=0.010661980015637305, P74=0.01173480910439261, P75=0.008989384671735129, P76=0.00840970077354736, P77=0.03564770087421315, P78=0.04706284619456555, P79=0.018568262924843315, P80=0.005676435896371905, P81=0.0028916219531565086, P82=0.0025156759595125486, P83=0.003112460835834524, P84=0.0060788698924359764, P85=0.009646969649971785, P86=0.005470438916937652, P87=0.002610936960890641, P88=0.004125064328048465, P89=0.012534766364153202, P90=0.01713228152919015, P91=0.010624643356559736, P92=0.00684615612547675, P93=0.0020767803880669834, P94=0.0006861241595392187, P95=0.0015108015508812785, P96=0.001557304721935361, P97=0.001899245899689914, P98=0.022395731845846627, P99=0.02223202450255696, P100=0.0012284558950232676, P101=0.0002178178575755541, P102=0.00019676080478800732, P103=0.0007566927370673024, P104=0.002752523557233002, P105=0.003396097875793797, P106=0.0028989871755748776, P107=0.0001452973805248437, P108=0.00026104099056839075, P109=0.00018632578564323093, P110=0.0002653229817870002, P111=0.0013776810320865604, P112=0.000539372765334031, P113=0.00011488998094088687, P114=3.3650706320893705e-05, P115=0.00040919158272019127, P116=0.0011420794964389091, P117=0.0002363980424173731, P118=3.370483070983452e-05, P119=0.0003202117044217532, P120=0.0002964101080093905, P121=5.587899160688018e-05, P122=6.021733558235743e-05, P123=0.000195644130671704, P124=0.00010858158991286003, P125=0.00011404098616650878, P126=0.00013068289366063374, P127=0.00010012616546539309, P128=8.9735961490191e-05, P129=0.00017708095058005196, P130=0.0003029055677223102, P131=0.0003411926703679688, P132=0.0005668834377592167, P133=0.0001838568135602108, P134=7.715833515110172e-05, P135=4.9349505261307116e-05, P136=6.436763058700327e-05, P137=3.389469411254103e-05, P138=3.7464323860822504e-05, P139=3.489568121272741e-05, P140=5.259418459862338e-05, P141=6.01022577429909e-05, P142=9.050350982788958e-05, P143=4.3905942720594615e-05, P144=1.864863610745421e-05, P145=1.887094064076769e-05, P146=1.65680568425216e-05, P147=4.228007744087055e-05, P148=5.499197583237563e-05, P149=3.475634684299726e-05, P150=6.505543413446009e-05, P151=4.8261848868447225e-05, P152=5.298767834539959e-05, P153=3.5483850040959793e-05, P154=2.749439160000665e-05, P155=4.3889685059752645e-05, P156=5.3090179393963485e-05, P157=3.246374379721246e-05, P158=3.2387166616070286e-05, P159=7.3046096652184e-05, P160=0.00011729314799635187, P161=0.00017156292728787406, P162=0.00024088304425957656, P163=0.000575897347277097, P164=0.0004275007904248644, P165=0.00046684347506503445, P166=0.0007467354267801781, P167=0.0008773580861919598, P168=0.0004612323212178126, P169=0.0003092177423247321, P170=0.0003177850190620452, P171=0.00032168517210441334, P172=0.0004173830403410126, P173=0.00043875839261681485, P174=0.000250718543676143, P175=0.00018110302010976295, P176=0.00011750518992963895, P177=9.651416400202112e-05, P178=8.154809787509078e-05, P179=1.1501879297501945e-05),Row( P0=-630.2898186765943, P1=39.12693870704455, P2=-11.825445183720902, P3=-3.0299462394541608, P4=-17.327411224355874, P5=-11.10518447140747, P6=-8.820326655152055, P7=-2.8513592019322633, P8=-4.565107609145576, P9=-10.373893148139153, P10=-8.279584436642523, P11=-1.557031437763211, P12=-7.9900954426583315, P13=0.6048520373546661, P14=8.133461770936119, P15=0.862667488710318, P16=3.322813132734945, P17=4.1742649661624345, P18=-0.2607164585871211, P19=1.3588629163878347, P20=1.8519560493735894, P21=3.9364963026154394, P22=0.8421247529234606, P23=-1.8767218039173437, P24=-3.6106696109241514, P25=-3.5608858000643044, P26=-0.9276059774898618, P27=0.7308066633462729, P28=-1.477561314238835, P29=2.1396084994582796, P30=-2.135216842905138, P31=2.2512527270834597, P32=-1.8211314671133745, P33=-0.2857052080635015, P34=-1.6341821512955585, P35=0.02541228764977516, P36=-1.9266854994143907, P37=-1.2913692493785085, P38=-1.4330717399681783, P39=1.4910640910967592, P40=0.4886752815935372, P41=0.5282052133276027, P42=0.5007778895169814, P43=0.4660844021697233, P44=0.43141673262855906, P45=0.3603350605378737, P46=0.3769741181272074, P47=0.4049134956849476, P48=0.4241437992643194, P49=0.4565859377065458, P50=0.4876719204507038, P51=0.4986226670749105, P52=1.171153715653446e-05, P53=2.5273407833119245e-05, P54=3.532023667408686e-05, P55=4.308101529114027e-05, P56=4.453233253277505e-05, P57=3.951598506279066e-05, P58=3.844819597013024e-05, P59=4.862128748007309e-05, P60=5.60797329607688e-05, P61=0.00019525086557807168, P62=0.0026303108744952456, P63=0.008704443598424285, P64=0.021764735480867447, P65=0.05024967630248863, P66=0.1471198357232387, P67=0.20643212568583744, P68=0.12090066901818117, P69=0.15187941247382403, P70=0.09713694944115017, P71=0.060516743947863376, P72=0.020370054650505197, P73=0.003943625762621446, P74=0.001298984695041667, P75=0.0018461042164929719, P76=0.004901087864456748, P77=0.019595031976794026, P78=0.029096592977154088, P79=0.02630923919302452, P80=0.016072381237811986, P81=0.012376978990160756, P82=0.01711430508927252, P83=0.02842457607604709, P84=0.05527032703005865, P85=0.11773029844401027, P86=0.1257844209614165, P87=0.0669351276030527, P88=0.03824833684593275, P89=0.05809444529677113, P90=0.09460718468364393, P91=0.07883073059842848, P92=0.013875205872253134, P93=0.006324866735577534, P94=0.00869578760463976, P95=0.008721516630834395, P96=0.0025769923678315913, P97=0.0016622243675982306, P98=0.002754010153167644, P99=0.003580923579024541, P100=0.00982244033400765, P101=0.015243034050607583, P102=0.07438272506265495, P103=0.08153449604234042, P104=0.008489295985162643, P105=0.0007163012498993889, P106=0.0008786235392945654, P107=0.0007065772326356949, P108=0.0006877482728938223, P109=0.002976177890861317, P110=0.014382539098382316, P111=0.04467847521231645, P112=0.03082180265012598, P113=0.0015267741135370778, P114=0.0005704280938637588, P115=0.0006621878746334474, P116=0.0013545360681647004, P117=0.002267973199224254, P118=0.002931067647356423, P119=0.0017126096350311888, P120=0.0006612152037565254, P121=0.0009196795754312043, P122=0.0013442417233545305, P123=0.0012403716836260307, P124=0.0016698452672333078, P125=0.0013805129545541403, P126=0.0004925715065783535, P127=0.00046636577410385263, P128=0.001022478895987726, P129=0.0006479017831962276, P130=0.00024996895146533424, P131=0.0002729986057716283, P132=0.00010855925261116116, P133=0.00010241652678113862, P134=9.742267052928756e-05, P135=4.046107181822554e-05, P136=5.655266443304749e-05, P137=4.911581252200802e-05, P138=9.12844081952028e-05, P139=0.00016627015365846542, P140=0.0003152898244051617, P141=0.00014573494109647195, P142=0.00020954072440858217, P143=0.00022211809631240607, P144=0.00016975102710326776, P145=0.0002841811649796661, P146=0.00030511642725031057, P147=0.00037467011803163136, P148=0.0005214237128286547, P149=0.0006238228463867139, P150=0.00043626190853171407, P151=0.00025140558948017264, P152=0.00012019733045605517, P153=6.650426844323436e-05, P154=8.629070623774654e-05, P155=9.522960597427954e-05, P156=0.00015160994659237396, P157=0.00010380276431708618, P158=0.0001414091403856141, P159=0.00024531873376090285, P160=0.0002851507551539179, P161=0.0002463248554760708, P162=0.00017740594028306295, P163=0.00015345786093194435, P164=0.0001829582761580439, P165=0.00013419016517504542, P166=0.00021414153389461745, P167=0.0002716439786816213, P168=0.00019543775854655518, P169=0.00014473919815127025, P170=4.665845100338297e-05, P171=2.9633593268094803e-05, P172=4.44726508046922e-05, P173=5.7328097538000474e-05, P174=7.750139016032805e-05, P175=7.870674878965561e-05, P176=7.003999589280003e-05, P177=5.9771419865259043e-05, P178=2.7560982320079134e-05, P179=2.501239175623852e-06),Row( P0=-442.9233135595193, P1=12.679741726591669, P2=-38.5415951805551, P3=-9.268446271357398, P4=-28.860322689881105, P5=-17.624857775421564, P6=-8.046678196130106, P7=-0.2839942079558455, P8=4.380696216704428, P9=4.376863424194269, P10=6.068460247261958, P11=11.133678085360184, P12=-7.111341292646689, P13=1.5029198832711808, P14=5.33382230467207, P15=-7.179083045712317, P16=-1.0814426603327991, P17=-1.345988761510619, P18=-2.716047462734958, P19=-3.636606365975965, P20=-0.48379680385549106, P21=3.324644285115235, P22=0.063718675473595, P23=-0.3931397187434837, P24=0.569178045316725, P25=-0.9123951828237314, P26=-2.3063800925000324, P27=0.44660630132411105, P28=-1.6731839717927648, P29=3.201389513265771, P30=-0.7990985259834031, P31=1.6268019394487565, P32=-1.1239732803646527, P33=1.3811623277095062, P34=-1.1463179842506261, P35=-2.145532178200724, P36=-0.7676302177821496, P37=1.7563735162673664, P38=-0.7262721991027844, P39=-1.4578783544639224, P40=0.6186199632127819, P41=0.620746940301114, P42=0.5259596066151456, P43=0.42242697560209275, P44=0.4302434441742734, P45=0.46172992622054737, P46=0.44098974497436033, P47=0.4463840020637267, P48=0.4691676740078108, P49=0.5406006736845747, P50=0.5797348148205179, P51=0.5822224321147635, P52=7.051911688933413e-05, P53=0.0001234310086061186, P54=0.00019790298559888716, P55=0.00018792620800421174, P56=0.00017444003111041976, P57=0.00017483818414316477, P58=0.0001616295863519859, P59=0.0002242396701879901, P60=0.000338918325113484, P61=0.00034252526793289396, P62=0.00035330591146879177, P63=0.0003236455406722367, P64=0.0002474362270897799, P65=0.0003800867659691706, P66=0.0009142486407124231, P67=0.001700267909806153, P68=0.018587583307515653, P69=0.1135158014519584, P70=0.109294391122775, P71=0.09629042379913638, P72=0.24972870501147013, P73=0.3624789277650443, P74=0.5443121786949086, P75=0.7729546685733302, P76=0.7459600259335001, P77=0.9918079595952861, P78=1.4923172862637446, P79=1.4550182283378734, P80=0.8504575473232115, P81=1.1813451745400565, P82=1.3551319278069078, P83=1.1806770705318643, P84=1.3276753959807877, P85=1.8718520467260722, P86=2.6497159472733123, P87=4.508431455522039, P88=2.499838093467828, P89=0.6730373010653125, P90=0.2782048143555604, P91=0.3045159968337838, P92=0.692725113756962, P93=0.6311698645207706, P94=0.6603979483593136, P95=0.5827443801634345, P96=0.22191661453960249, P97=0.15092854090603894, P98=0.14019263620113662, P99=0.2562837409166124, P100=1.1741485003547147, P101=0.9632460972404734, P102=0.3393136048334294, P103=0.21647302131270152, P104=0.6730033588560974, P105=1.3419729462776584, P106=0.3704736637772771, P107=0.06728231636417202, P108=0.058586770768844894, P109=0.05751515956824252, P110=0.051984186386247515, P111=0.10660841287421588, P112=0.6393189933012371, P113=3.406816235454208, P114=1.0808073148081883, P115=0.2825937954477249, P116=0.08620373874855733, P117=0.04318751676904318, P118=0.10575189909279088, P119=0.08659817646381843, P120=0.10617801100451402, P121=0.2822490208562973, P122=0.23497582381165508, P123=0.05704669063600179, P124=0.030210869559005517, P125=0.058824732395999835, P126=0.09822220232292989, P127=0.140104948318863, P128=0.15026895433751897, P129=0.07895366678890622, P130=0.041814672308054725, P131=0.027279132130779458, P132=0.021306489626778165, P133=0.02196692744570182, P134=0.020474681813683875, P135=0.038485859941556795, P136=0.03620794666374527, P137=0.01559270317931791, P138=0.025439596736510996, P139=0.025301502556079707, P140=0.018343261745805903, P141=0.020326746610391463, P142=0.02086436286777735, P143=0.012162291539809739, P144=0.011760060970206928, P145=0.01526389601912292, P146=0.013188674669009873, P147=0.018827970180283156, P148=0.02059363602629343, P149=0.016838849458001858, P150=0.011733547995246755, P151=0.003451788018628587, P152=0.0017647712732515482, P153=0.001528209107910029, P154=0.0019954939880314267, P155=0.00215174542676796, P156=0.0021251611982808227, P157=0.0017035773206751433, P158=0.00242006576448276, P159=0.0035904552397064775, P160=0.0033587777021325655, P161=0.0024041137006968437, P162=0.0028681893346816797, P163=0.003035072054139145, P164=0.0026657899481367154, P165=0.0024404608836496667, P166=0.002812952455966205, P167=0.002292922309448987, P168=0.0013602181397438572, P169=0.0008933945232614917, P170=0.0008139663281529108, P171=0.0006599223765044706, P172=0.000879873514079385, P173=0.0010921529521439903, P174=0.0012114608266674213, P175=0.001123532191501268, P176=0.0007953518150097107, P177=0.0007009476672082179, P178=0.0004024874125760646, P179=6.170956516251621e-05),Row( P0=-693.7944296002395, P1=45.22767091238179, P2=-4.614745528947279, P3=-8.939226409098698, P4=-19.77549074086761, P5=-15.9648931336573, P6=-4.215825347582108, P7=-4.364057809215271, P8=-9.795691201137231, P9=-6.780398947859101, P10=-7.184692607944728, P11=-4.661909193044259, P12=-10.937690969239759, P13=-3.0584993666592273, P14=-5.515285608842316, P15=-5.166696528388597, P16=1.5742987563168203, P17=0.40603200332169787, P18=3.46903448744043, P19=4.44306852305535, P20=4.4796555021056905, P21=7.027206723695344, P22=1.8353585897754248, P23=3.408654784765557, P24=2.0102695113245637, P25=5.062955201377729, P26=2.4268407793241358, P27=1.2844285561781401, P28=-0.7505621458783391, P29=-1.7405658916888538, P30=-2.9143489095092905, P31=0.8356179508198235, P32=0.9370027378468703, P33=0.6579700845602203, P34=-1.2909445662245462, P35=-1.4291500314065773, P36=-2.635672336977603, P37=-0.8054039819498593, P38=-1.1829969430720448, P39=-0.02143212050426591, P40=0.5599452724548342, P41=0.5297199995980552, P42=0.5305233927074016, P43=0.5747372067569125, P44=0.6496666729635022, P45=0.5722480258992757, P46=0.5051778638945741, P47=0.5367404745891905, P48=0.5520386582740722, P49=0.5458757392818294, P50=0.5549869731150268, P51=0.5493073982893117, P52=1.0915787280819417e-06, P53=1.8779614044088158e-06, P54=1.8632697424064856e-06, P55=3.6465034388710198e-06, P56=4.29467380120164e-06, P57=3.1064457174799684e-06, P58=2.6203568624846697e-06, P59=8.525638972025558e-06, P60=0.0009081067892719145, P61=0.009225926910019782, P62=0.012180201574638819, P63=0.008028720137369771, P64=0.009945395404937676, P65=0.01848541682332338, P66=0.022568141874044236, P67=0.010245312536864691, P68=0.003733856634725845, P69=0.0005000814649963672, P70=0.00019420832328083753, P71=0.0010784356745352404, P72=0.0027587274030348055, P73=0.002333180458642473, P74=0.002736141300383292, P75=0.004473454885684426, P76=0.03234211945421288, P77=0.05660458487322388, P78=0.02036231439257014, P79=0.015623097615258605, P80=0.015707576734312317, P81=0.010794467492194737, P82=0.004759685734215535, P83=0.004401472981583173, P84=0.009054484749402304, P85=0.008942763719758629, P86=0.0017504394145013025, P87=0.0006118598676932636, P88=0.002111022312011781, P89=0.005460686309809264, P90=0.006551122366723346, P91=0.009795348193070751, P92=0.018707942244238832, P93=0.01849171981489854, P94=0.00518014248781951, P95=0.0005700035339837755, P96=0.00013367352481910599, P97=0.0005365338970643418, P98=0.0013255999036599392, P99=0.003436651142926904, P100=0.012790465894736001, P101=0.041362161462193424, P102=0.03384934320625011, P103=0.0030636029856191908, P104=0.00018008827392935193, P105=0.00022395015854748124, P106=5.447186962860226e-05, P107=0.0005759060610463795, P108=0.0009363233257137887, P109=0.00021881406469995786, P110=6.47917939886463e-05, P111=2.063855260292511e-05, P112=5.55359089704764e-05, P113=0.001186615533034254, P114=0.0018150996187761596, P115=0.0011452796946127492, P116=3.0223535208601662e-05, P117=0.00023423698902230055, P118=0.00032280102272265964, P119=0.000215803247470527, P120=0.0001347286645209249, P121=0.00010121249299927359, P122=9.629900184415315e-05, P123=5.9086748329876035e-05, P124=5.8539028494791484e-05, P125=5.3998733420806855e-05, P126=7.939835709804501e-05, P127=0.00013140154584730926, P128=3.6663667882148564e-05, P129=5.9129528447385625e-05, P130=9.54841040484112e-05, P131=8.580449617782233e-05, P132=0.00019256969012057927, P133=0.00023317004322704845, P134=0.00010582096505379343, P135=5.751249524421377e-05, P136=3.1315014267166984e-05, P137=3.0257205829722615e-05, P138=2.9811586163253545e-05, P139=1.9114567982121517e-05, P140=3.1886214378872245e-05, P141=2.277367481277754e-05, P142=2.346069407609258e-05, P143=2.0722404431279738e-05, P144=2.4641587903254345e-05, P145=3.8385446953110033e-05, P146=3.807465736038464e-05, P147=5.974440454797094e-05, P148=5.8408277958778994e-05, P149=6.419911123741017e-05, P150=6.793282694685904e-05, P151=7.893334011724089e-05, P152=5.51636726502978e-05, P153=1.9185409058670536e-05, P154=1.3481020400629035e-05, P155=1.4016749756448057e-05, P156=1.8621269481440394e-05, P157=3.330873570278155e-05, P158=0.00017118231224784564, P159=0.00023389687217449429, P160=0.00019713608981462155, P161=0.0002240446855260617, P162=0.0002929893043687736, P163=0.0003683607202358331, P164=0.00023327007503516175, P165=0.00018381326079685922, P166=0.00018150731821025935, P167=0.0001223069670131483, P168=0.0001604281361736216, P169=0.00022005140056567598, P170=0.0004298997523758937, P171=0.00036146635856683005, P172=0.0002520613677993393, P173=0.00020068880861319043, P174=0.00012447856208735322, P175=0.0001775428919836582, P176=0.00020612353468567616, P177=0.00019652682279996063, P178=0.00010535906657531153, P179=5.831838413691489e-06),Row( P0=-618.0154363478763, P1=37.900185725894524, P2=-12.558327734717142, P3=-4.187784027107057, P4=-19.88661750515729, P5=-14.112640562692867, P6=-5.042863880733099, P7=-0.6198546681844072, P8=-7.622352361767224, P9=-11.28111019852466, P10=-6.6492366209039675, P11=-1.6732564007748345, P12=-8.950185533675597, P13=2.3051306819158075, P14=5.16749787759619, P15=-0.4614061507548837, P16=4.126181146333406, P17=2.1904314532592153, P18=1.3882655986955919, P19=-0.6272715809522968, P20=-0.633679529106913, P21=3.0989894792406445, P22=-1.9462792522418422, P23=1.0909538337571612, P24=-1.2681973688182298, P25=-0.988450954517548, P26=-0.6462709467844486, P27=-3.0190540352292494, P28=0.30696133603365344, P29=-0.009184266066339172, P30=0.3642310839498355, P31=1.9337441245631575, P32=-0.05802414843394533, P33=1.2516739038606788, P34=-1.8357423747911215, P35=0.06590130431598488, P36=-1.8596761999672209, P37=0.0008280460911207191, P38=-0.3296524726024204, P39=0.5669369537442421, P40=0.7093097215107826, P41=0.7624798552081207, P42=0.6872363991281522, P43=0.6016150696082977, P44=0.5881014978475327, P45=0.5637705412460966, P46=0.5236075097865782, P47=0.5308036233335074, P48=0.5599561991368384, P49=0.5675538002303591, P50=0.665293493270804, P51=0.6834598153642307, P52=9.333675944994274e-06, P53=1.355292886992701e-05, P54=1.997840577355159e-05, P55=2.961575944894066e-05, P56=4.122452335420834e-05, P57=3.833102396980991e-05, P58=3.0957726740375224e-05, P59=0.0001247462372833292, P60=0.0005892346465368524, P61=0.0024834563143020785, P62=0.010980893478915253, P63=0.012108553364650092, P64=0.018204730257782693, P65=0.11747697205904953, P66=0.17211107160682862, P67=0.14448597867467378, P68=0.12558016278623949, P69=0.05046141803330661, P70=0.03369579318866299, P71=0.0759321706908534, P72=0.04469193282452583, P73=0.018319045673140487, P74=0.0101929345131566, P75=0.007196392998585365, P76=0.005865193222122537, P77=0.006962247824662047, P78=0.0063946762126340225, P79=0.020967543096645492, P80=0.050832474306497705, P81=0.027616131219535177, P82=0.016863691815513265, P83=0.02235532039941139, P84=0.06240257013953299, P85=0.15001293287902637, P86=0.0905355645831776, P87=0.04382125087304721, P88=0.03939641240975854, P89=0.06844486301102971, P90=0.08074795083539077, P91=0.05611293041333231, P92=0.056590766882693044, P93=0.025803437477707535, P94=0.020889542535927622, P95=0.017106147564843625, P96=0.02154811903462517, P97=0.1714373526422729, P98=0.15310954257008635, P99=0.00857200102806278, P100=0.005519486832826837, P101=0.01546056485801721, P102=0.008742078691355945, P103=0.0026862499244168295, P104=0.0024403648881088018, P105=0.0069599112669923545, P106=0.019130935656012704, P107=0.006204072507964444, P108=0.00041163188903472955, P109=0.0005435296074384309, P110=0.00041767346304381805, P111=0.0008764124598585205, P112=0.0015275175217112778, P113=0.018155878629909094, P114=0.01247739880844208, P115=0.0006783613704270851, P116=0.00017630328472940734, P117=0.00045235591306553436, P118=0.0015348272585050727, P119=0.0013106638718302255, P120=0.00017806729322533856, P121=6.232956741212672e-05, P122=0.0007836042447970125, P123=0.0016490855878630172, P124=0.0010738515073871234, P125=0.0004150200388244816, P126=0.0020751389926200684, P127=0.00048191647843328023, P128=0.000412834400246391, P129=0.0011268138205720523, P130=0.0007263819459531763, P131=0.00028049549619718655, P132=0.0006140391231858144, P133=0.0007530887157887458, P134=0.0002583551742989898, P135=0.0003066009022469841, P136=0.00014546924699620803, P137=0.00024701738596151073, P138=0.00018053698054282746, P139=0.00033381723868478643, P140=0.0005148122338974715, P141=0.00015129863487529772, P142=0.00016459581913345835, P143=0.00012736106160359163, P144=0.0001669174822756565, P145=0.00022125708447011394, P146=0.00010981731048961794, P147=0.00020062597486194544, P148=0.0001383112819968142, P149=0.0005361117320477177, P150=0.0002559675166089423, P151=0.0003731491348865127, P152=0.00035046813214322393, P153=0.00019056728397293216, P154=0.0002455282570419657, P155=7.470728112315203e-05, P156=0.00011197256904437814, P157=0.00011664129483366102, P158=0.00012705453051673878, P159=0.00029905252834786926, P160=0.0005861174851137481, P161=0.000548000130241457, P162=0.0008219625308332983, P163=0.0009839967469138621, P164=0.0012783442129834068, P165=0.001032551550972817, P166=0.000496854659539146, P167=0.0003105179133838398, P168=0.0003759550491632257, P169=0.0008719387948918378, P170=0.0021360861009003776, P171=0.0028900750350016138, P172=0.001470291799748198, P173=0.0009488247415024339, P174=0.0004482739218807931, P175=0.0003959548476143785, P176=0.0005611859245455629, P177=0.0005075564647969436, P178=0.0002789253677843028, P179=4.233197784003274e-05),Row( P0=-628.0215526180431, P1=35.80625555770035, P2=-22.82040933567747, P3=-2.2064256109243123, P4=-19.307994693341925, P5=-13.20233336336776, P6=-6.422908697043542, P7=-4.8386543675791645, P8=-7.757090093902365, P9=-10.273867022258731, P10=-8.489895762827468, P11=-2.806118018126697, P12=-10.701720785425385, P13=-0.6426516803796183, P14=-0.3518290505921307, P15=-2.653238353221386, P16=1.739764293311596, P17=0.3804588926067569, P18=4.198713533806769, P19=2.313617310363332, P20=3.462620061705679, P21=5.602533333015758, P22=2.3557461467669114, P23=0.5338284902858257, P24=-3.7472215751662192, P25=-3.784453493924114, P26=0.06493073237618532, P27=1.9237729255883855, P28=1.2916465415133433, P29=0.33724680045324434, P30=-4.6642166090333435, P31=-0.22727648555916743, P32=0.8336154790600044, P33=2.4501417772179495, P34=-1.7060035555520716, P35=-3.083792551783392, P36=-2.7255897802098374, P37=0.10498619987465183, P38=0.10738684601942565, P39=0.6586443233140266, P40=0.4899207622185241, P41=0.5081956357009088, P42=0.5352349033659575, P43=0.5017356483397607, P44=0.5082488078675954, P45=0.4843859080245981, P46=0.42550383154685767, P47=0.41296598817733426, P48=0.5084004937433434, P49=0.5505267585086371, P50=0.4923454589596264, P51=0.48197637662557147, P52=2.2589790608211524e-06, P53=2.52478510453662e-06, P54=3.242097008471263e-06, P55=9.80189520285833e-06, P56=1.9477930000249757e-05, P57=2.4977990255699592e-05, P58=1.1753242938123134e-05, P59=2.0917629319561486e-05, P60=0.0005579702811687591, P61=0.004271835173125555, P62=0.00912202728129594, P63=0.023776949933902457, P64=0.04380747235582811, P65=0.038322436165508546, P66=0.0270389802333292, P67=0.027889361243517187, P68=0.02918139477210018, P69=0.058274734262191506, P70=0.04418826242085774, P71=0.010560704092302513, P72=0.0038224835454133493, P73=0.002429973263249581, P74=0.003659860233412214, P75=0.006639187206778206, P76=0.02332368246325707, P77=0.11768926354790821, P78=0.11540278057076728, P79=0.08041022590024419, P80=0.05133173474713284, P81=0.033738310391359556, P82=0.016490445198071375, P83=0.006008724562964871, P84=0.006040894087119462, P85=0.031012387316644234, P86=0.09418604130037399, P87=0.10435066415247332, P88=0.13842751700021905, P89=0.11227789731405431, P90=0.07822950190661086, P91=0.11182406428994356, P92=0.054951213779090104, P93=0.008461497300534323, P94=0.006648914303889799, P95=0.006955744467845704, P96=0.003491905662263235, P97=0.03884517720193558, P98=0.14593797617253906, P99=0.0903695478824868, P100=0.02776140900441887, P101=0.0071768865628575595, P102=0.0009587831521870701, P103=0.00033085975007818736, P104=0.0012721361804504763, P105=0.02190532785801635, P106=0.08414405569417036, P107=0.08962715874590442, P108=0.057308415119283154, P109=0.0077060021629896894, P110=0.002289969916933629, P111=0.0034250188571521324, P112=0.002928188130048849, P113=0.006514647189429258, P114=0.011334316323854425, P115=0.020143255372146784, P116=0.004814218743511062, P117=0.0009143929484007063, P118=0.00235414473332264, P119=0.006635225644117558, P120=0.009720117981547418, P121=0.0037702626547241715, P122=0.0052914416722906945, P123=0.006208305784214388, P124=0.004851975761003442, P125=0.0037163598681274534, P126=0.0033343044234639884, P127=0.003660684699331393, P128=0.0012995163763092096, P129=0.004524349197296903, P130=0.0029109795971748384, P131=0.0018338429938552013, P132=0.0009085660233418707, P133=0.0003851891455933071, P134=0.001035235016334036, P135=0.0005639358068127407, P136=0.0002696221120938349, P137=0.000470590716247609, P138=0.0003958108326560726, P139=0.0012735752444703748, P140=0.0006684427506524643, P141=0.0007393916291152374, P142=0.0004640571675324875, P143=0.00036176207646314567, P144=0.00027594173123378466, P145=0.0002778798546015526, P146=0.0005438517757335299, P147=0.0006354593211595709, P148=0.0009274058937533715, P149=0.0013530405837131774, P150=0.0011900646154027385, P151=0.0006456539332817689, P152=0.0002736782549704143, P153=9.077601125123395e-05, P154=3.830436105090525e-05, P155=0.00016551249295962363, P156=0.0002634046669669105, P157=0.00035376991182013435, P158=0.0005684949441479992, P159=0.0005086224976909987, P160=0.00032027632332055027, P161=0.0002582348759514699, P162=0.00015542631114695506, P163=0.00015025531064445001, P164=0.00028926228896561975, P165=0.0002080784362613564, P166=0.00021490241529081413, P167=0.0003503465246363042, P168=0.00032759997530716997, P169=7.293564353036886e-05, P170=4.441552077211284e-05, P171=7.771190941214492e-05, P172=0.00014938865250760833, P173=0.00016337056656476264, P174=0.00012780316627743627, P175=0.00012167808486767907, P176=0.00012030962121133433, P177=9.526678426684962e-05, P178=5.3897224593465336e-05, P179=8.252574511993827e-06),Row( P0=-552.2046420266933, P1=22.90579069311426, P2=-22.141446876381366, P3=-9.43460859726841, P4=-25.33670706151872, P5=-17.193924049926764, P6=-7.349414593581974, P7=-7.120815072347917, P8=-11.46516763245446, P9=-11.198848682396559, P10=-8.873410082057948, P11=-0.7698318911143458, P12=-6.787130665714384, P13=5.746329424001047, P14=7.087833929907457, P15=5.631423542142116, P16=4.828173111756328, P17=-0.7663874944055626, P18=-0.05356334822782144, P19=0.49053664937186403, P20=1.6858398688933847, P21=2.432618752754107, P22=-0.11110809304614401, P23=0.7884903801471492, P24=-0.6179882810964336, P25=-2.3335847672961183, P26=-2.9411685699857966, P27=-3.3301193851143482, P28=-1.2657124137246076, P29=3.971408414307284, P30=0.25022122560548515, P31=1.7659311542022051, P32=-1.1962262052601336, P33=0.6759242535310097, P34=-0.16562493485484764, P35=0.0011729318293580867, P36=-2.075341300757931, P37=-0.6812795336129258, P38=-0.8240692952167824, P39=1.6171657127027195, P40=0.659190599548112, P41=0.6790625679592965, P42=0.655998844041081, P43=0.6337183104802514, P44=0.5821541044673296, P45=0.5553243718048511, P46=0.5619077559398812, P47=0.5899268378922131, P48=0.5755187110233021, P49=0.5356416510624107, P50=0.5383539680426198, P51=0.5813304977251148, P52=4.067739334805773e-06, P53=4.045727191039458e-06, P54=1.944795085822863e-06, P55=5.190434114845133e-06, P56=9.064332668715502e-06, P57=1.684295618194477e-05, P58=1.6227351706637703e-05, P59=2.011691609261095e-05, P60=4.886424693969951e-05, P61=0.00030490808950656953, P62=0.0073559251025140686, P63=0.018040449139094982, P64=0.017486733311792974, P65=0.031524831515598296, P66=0.09618203525357004, P67=0.1819302030512153, P68=0.21391550974680676, P69=0.4059141492047911, P70=0.3936793817370698, P71=0.14462709294986387, P72=0.10796714722171419, P73=0.14078681491576792, P74=0.33917583418578306, P75=0.3742425858608187, P76=0.13801248561691226, P77=0.02992658450903669, P78=0.027311023641340804, P79=0.03324045441808278, P80=0.035398851547496196, P81=0.043167927873792124, P82=0.125468529032523, P83=0.35102994518364194, P84=0.6272724628773214, P85=0.8589980429558384, P86=0.3891747599645734, P87=0.13095492743766754, P88=0.11442441667549895, P89=0.24163097146821566, P90=0.18679545464012007, P91=0.18570224029823354, P92=0.17821386253538565, P93=0.08221335336208709, P94=0.0685869476191827, P95=0.08714785091819965, P96=0.07695760430214893, P97=0.16227895776769385, P98=0.22275888197163177, P99=0.06984356951277645, P100=0.009911063142488788, P101=0.14240696762414423, P102=0.37002118687561514, P103=0.11567711339084669, P104=0.16927381262977526, P105=0.17715879143068455, P106=0.05146452996359169, P107=0.007345929723320927, P108=0.001155305687804537, P109=0.00066570567601393, P110=0.012979070955261534, P111=0.026727320804417396, P112=0.025517190348122065, P113=0.013043095797596112, P114=0.019926646111163716, P115=0.006380760122612063, P116=0.011546969803339871, P117=0.051246497951468094, P118=0.012733550001938338, P119=0.010998118961164392, P120=0.007908244938667703, P121=0.014331266450941283, P122=0.023027015278780795, P123=0.011285176064256686, P124=0.005959120690986144, P125=0.005501986626155335, P126=0.005844514109030997, P127=0.012783210111126797, P128=0.01254712207843582, P129=0.0037295618140723826, P130=0.0038511873627970978, P131=0.0037191917350377804, P132=0.0047286940367859106, P133=0.005872301705214027, P134=0.005091377968587826, P135=0.00253269638289357, P136=0.002438366765284141, P137=0.0013608486359641412, P138=0.004178600063835517, P139=0.004059948996087038, P140=0.0017809063067277511, P141=0.0028249216215033516, P142=0.002366532452896223, P143=0.002430461789211728, P144=0.004545053372088049, P145=0.0056291562112933445, P146=0.004320267471829097, P147=0.004943405271842268, P148=0.004326731961791887, P149=0.007721800617553836, P150=0.00519746897470164, P151=0.0024644295116634162, P152=0.0006815930826220207, P153=0.0003882580525305531, P154=0.00025743882311332964, P155=0.0002993553625884447, P156=0.00035523355664901946, P157=0.00041159125513264455, P158=0.0005539084151217176, P159=0.0005462010121987956, P160=0.000762117556275741, P161=0.0013310955522798284, P162=0.00201440166013443, P163=0.0018745346931695517, P164=0.0022519890086626047, P165=0.0022576682102360356, P166=0.0034306860749467477, P167=0.003039447764883291, P168=0.003320928159918492, P169=0.00279028176363863, P170=0.0014607691318884676, P171=0.0018866616476166075, P172=0.0024241663203679246, P173=0.0030829732023350737, P174=0.003971947041712471, P175=0.0027155981954577212, P176=0.002224054650836987, P177=0.001597675383789531, P178=0.0006483167828053642, P179=6.499372258960922e-05),Row( P0=-636.5376202919805, P1=54.56165813714294, P2=-12.112490558516017, P3=-1.912078948699417, P4=-14.646645338487774, P5=-15.681352548393432, P6=-4.55644774915593, P7=-5.3882691780167065, P8=-9.073715420964962, P9=-7.848324936909632, P10=-4.801788674863686, P11=-3.048339982528526, P12=-11.900649884624924, P13=-4.3690801114908595, P14=-3.4263868213847055, P15=-4.947954946539119, P16=1.3699518956667092, P17=2.6508670423335454, P18=5.649499493114749, P19=6.072168182082276, P20=4.6469524554845565, P21=6.404926697533764, P22=1.0899614846646053, P23=2.3607270378443386, P24=0.26590395148595175, P25=-2.0483223129277865, P26=-4.8587463077843065, P27=-5.262813213564582, P28=-4.305745208925172, P29=-1.352199118954739, P30=-2.802789612180536, P31=0.9329557328459914, P32=-2.82015038159685, P33=-1.7438611345545112, P34=-3.34554695708076, P35=1.4053377092896313, P36=-0.994383998126184, P37=-1.8938170607338494, P38=-4.305645162409053, P39=0.4884695894273347, P40=0.6540547672107334, P41=0.599889306304376, P42=0.5847051991804467, P43=0.5730586395419062, P44=0.5606697541994828, P45=0.6053865644856917, P46=0.6331007195955137, P47=0.6182226301952959, P48=0.619274672032894, P49=0.6150097602735725, P50=0.6774087387923999, P51=0.7213551373506477, P52=3.6696754370479263e-06, P53=4.638490483748337e-06, P54=1.240191610617811e-05, P55=3.057326968204429e-05, P56=5.093663750055377e-05, P57=6.042702871426928e-05, P58=2.657767323651389e-05, P59=2.3520939557778633e-05, P60=0.00014638754533852905, P61=0.0019313093991031084, P62=0.015004464720018696, P63=0.02603460640303276, P64=0.025101845259452926, P65=0.038355065974149595, P66=0.11303336067723138, P67=0.14015076282042424, P68=0.051273858201432154, P69=0.008400215821373243, P70=0.001936695143665447, P71=0.0003572187879703597, P72=0.001134153043101481, P73=0.003271675915165341, P74=0.00979479683065885, P75=0.021515184443135552, P76=0.04078876316339248, P77=0.05189820662408949, P78=0.042105774690594545, P79=0.022866131313954943, P80=0.024491691568053926, P81=0.06533205838357395, P82=0.07510853301150684, P83=0.022789970185022893, P84=0.013495233590029585, P85=0.010827809940844663, P86=0.0077117958601518955, P87=0.013234396443824363, P88=0.04597774031176991, P89=0.02926403286724019, P90=0.019677227885179546, P91=0.010132781894860868, P92=0.0029251848092388907, P93=0.01001954651419413, P94=0.02672630008448346, P95=0.020263533016054287, P96=0.019118987502577643, P97=0.039704111377616494, P98=0.02096868530502958, P99=0.0035363398565160763, P100=0.0022616665087734533, P101=0.002397111432402222, P102=0.026747122555492792, P103=0.05806679526810199, P104=0.029185470094466492, P105=0.00414902318180391, P106=0.0007368608594511793, P107=0.00037349905670382497, P108=0.0007309980359730284, P109=0.012295137039643463, P110=0.01689731871696602, P111=0.0005347836183143367, P112=0.00026649061864479384, P113=0.00195209450815278, P114=0.002490948004535948, P115=0.014946457624114966, P116=0.007689097840227349, P117=0.001782140350683542, P118=0.004426063061358096, P119=0.0018504198346639999, P120=0.0009382454384566438, P121=0.001116462295896782, P122=0.0027278282257583086, P123=0.0006734947485065526, P124=0.0006776223153014675, P125=0.0007150095535266819, P126=0.000995579955148164, P127=0.000762366830769859, P128=0.000770050832133822, P129=0.0006111125489063224, P130=0.00041733938188875204, P131=0.0006183878064839629, P132=0.0002979268242900973, P133=0.00039122117826580313, P134=0.0006012495375258773, P135=0.00013711557713107672, P136=0.00016592652598635446, P137=0.0001951514112102028, P138=0.0001417727567541037, P139=0.00036590516461020635, P140=0.00023793739805152457, P141=0.00025915334315402026, P142=7.991809481257967e-05, P143=7.843949800182103e-05, P144=6.195501438913712e-05, P145=5.4793358533905834e-05, P146=0.00010237639188471802, P147=0.00013122694957915524, P148=0.00010463189951772167, P149=0.00011641358484013647, P150=0.00011480582768652999, P151=0.00014118337599981052, P152=7.3346229626682e-05, P153=2.1466847760382345e-05, P154=1.664348428485505e-05, P155=2.772302369125629e-05, P156=3.616418813759072e-05, P157=1.3522517000378803e-05, P158=2.9853326716204608e-05, P159=5.65437678622424e-05, P160=0.00010114427102958932, P161=0.00010990310425271395, P162=8.187559244665227e-05, P163=6.836764783688953e-05, P164=9.947509046256128e-05, P165=6.914045892364199e-05, P166=4.941070797177536e-05, P167=1.9156233858191873e-05, P168=4.881205958703139e-05, P169=6.809080231047464e-05, P170=0.00010153865252529206, P171=9.546093163455199e-05, P172=0.00011161530704463567, P173=9.408921354768827e-05, P174=5.686700437470307e-05, P175=0.0001017699659940596, P176=9.259843318244676e-05, P177=7.481647974364293e-05, P178=3.4515395276238225e-05, P179=4.453362717597517e-06),Row( P0=-571.6133932324773, P1=34.0612934622034, P2=-13.675717022460729, P3=-10.756515635281236, P4=-20.373763776150984, P5=-14.171647004543862, P6=-4.649376130826943, P7=-6.633761850964278, P8=-8.371441192553059, P9=-7.995530545892468, P10=-10.06344018535943, P11=-3.200241522770342, P12=-10.36806821065785, P13=-2.3175274274788054, P14=-0.2214243651595328, P15=0.6224882629573353, P16=6.487109080111286, P17=7.739984439120052, P18=7.91970609134638, P19=3.2145476317992174, P20=2.4925130905676616, P21=1.0565693974173411, P22=-1.4217315503421264, P23=0.4960548195069255, P24=-2.3636372898267486, P25=-2.7436656921089733, P26=-4.06473036041626, P27=-3.574021804227312, P28=-2.6253295709450346, P29=2.579538200326154, P30=0.5946473661230517, P31=0.540110319113729, P32=-0.28528852577874564, P33=0.4296798718981357, P34=0.6517697816683539, P35=0.30685090546436383, P36=-1.2612987733019587, P37=-1.2903581683912366, P38=-2.150260251213295, P39=1.0938845130289643, P40=0.6134528221731234, P41=0.6200459859203147, P42=0.5857461440139049, P43=0.5492172489065474, P44=0.5160748485338722, P45=0.5048929812311013, P46=0.5353092824930519, P47=0.5339503084788912, P48=0.5442287817828934, P49=0.5684339714222065, P50=0.5876312814728623, P51=0.614520158783487, P52=3.93219824453401e-06, P53=8.238851685925491e-06, P54=1.3181883980682048e-05, P55=3.077786388781848e-05, P56=5.2235455117302925e-05, P57=7.406867675170074e-05, P58=5.164947680436587e-05, P59=6.43632842772657e-05, P60=0.0002630940728157251, P61=0.0013516124836506532, P62=0.007928774604774782, P63=0.022350356891366687, P64=0.0357058726302824, P65=0.07806252794670228, P66=0.20526395143238033, P67=0.26784134808161203, P68=0.26669118626645794, P69=0.3489047853159341, P70=0.10798708533101818, P71=0.003733674490907413, P72=0.0006996333909457977, P73=0.0010415675539484023, P74=0.004793681962959036, P75=0.014204184403112548, P76=0.04950484635507746, P77=0.08747816480110972, P78=0.08457823660164934, P79=0.09981763074131707, P80=0.11343671079736146, P81=0.11671432246486316, P82=0.2807217389891086, P83=0.5745719830747, P84=0.23511307154764374, P85=0.08162696675068253, P86=0.039550473565086314, P87=0.025540060904197307, P88=0.055280521428952685, P89=0.052053849457871046, P90=0.04260515291198606, P91=0.07064684579066548, P92=0.07811451427069982, P93=0.03271715950255011, P94=0.08242279840849114, P95=0.1240802919714249, P96=0.04514149535186798, P97=0.024954087891364135, P98=0.04001998244928564, P99=0.01948555328106527, P100=0.010949049104243956, P101=0.011843601669327944, P102=0.009171078265438127, P103=0.005361072883792936, P104=0.007859575050182916, P105=0.006903419764397969, P106=0.003284395658342823, P107=0.0018620731900695388, P108=0.0018629292852387542, P109=0.005127395731255651, P110=0.009563593988953454, P111=0.013763644158522596, P112=0.003940457078909084, P113=0.0008192691420784274, P114=0.001187725178989121, P115=0.001427913353083003, P116=0.005017766152951245, P117=0.0024405226289495683, P118=0.00033722383290077693, P119=0.0007895814899833138, P120=0.0012804488639748847, P121=0.0023475893287769776, P122=0.0009804290770332438, P123=0.0015730661013789616, P124=0.0016354771345698928, P125=0.002115632598278324, P126=0.0016238991363031296, P127=0.0007871665854164985, P128=0.0011437045451089154, P129=0.002719636600309267, P130=0.00047398905570131787, P131=0.0007091173409497313, P132=0.006620285234220841, P133=0.00285639373412693, P134=0.0004397020630795753, P135=0.0019280691353401813, P136=0.0028306159913041506, P137=0.0007835463425088304, P138=0.0008353787554899642, P139=0.001524141436231148, P140=0.000870707456640796, P141=0.0008564281466514807, P142=0.001367866898638127, P143=0.0007222908705665943, P144=0.0007110660128442553, P145=0.000683195578064517, P146=0.0006770590035015771, P147=0.0006777361266720227, P148=0.0005151926669157273, P149=0.0006954718344033639, P150=0.0007262601054860526, P151=0.0005518699431680246, P152=0.0004833925354976074, P153=0.00021775318380644148, P154=0.000285111103124753, P155=0.0004617983137000123, P156=0.0005247575060253442, P157=0.0007198473523862119, P158=0.0007971761779092235, P159=0.0004972125353902297, P160=0.001018283763268421, P161=0.0014565588347661958, P162=0.00317533762939873, P163=0.004415653273148119, P164=0.008090202360572406, P165=0.009942442049052527, P166=0.008581596081186128, P167=0.00960004484184274, P168=0.004725589134332575, P169=0.0040006048728610365, P170=0.004604034273432184, P171=0.004795909044271513, P172=0.005162074875874164, P173=0.003948935398470927, P174=0.003938900244289087, P175=0.0031313826436921008, P176=0.002036447797379788, P177=0.0020173918787881376, P178=0.0014970113244606206, P179=0.00012084476511384629),Row( P0=-437.9059827058597, P1=9.683932971098276, P2=-38.96763291342335, P3=-11.488374178205339, P4=-25.774997346948847, P5=-17.672799117649568, P6=-2.409613195449828, P7=2.8697492930631308, P8=8.876475235007767, P9=4.892543509775528, P10=4.599730005772706, P11=10.579606451014488, P12=-9.774371627842875, P13=-0.9144575465875799, P14=2.2263753197696254, P15=-7.197914364776828, P16=-2.4104575493095353, P17=1.801905753091348, P18=-2.658733604957832, P19=-2.9412083609507467, P20=0.5470784199634339, P21=5.6750899818005, P22=2.860744851933778, P23=1.5874297907114185, P24=-0.7351919661615667, P25=-0.9796936114826216, P26=-1.0653065562522501, P27=1.7109897837641814, P28=-4.628553868453268, P29=2.2784134279973296, P30=0.27640679611669844, P31=1.8211346260952919, P32=-2.1668814933007394, P33=-0.3704985973500959, P34=-0.16849728648086296, P35=-0.28488101419906736, P36=-1.9147192936454156, P37=1.1114466122985494, P38=-1.173647080580453, P39=0.9525965174524296, P40=0.6708694033893302, P41=0.6776838338954508, P42=0.607711033387981, P43=0.5281745677862009, P44=0.4818434600990538, P45=0.45792978346123037, P46=0.45222137595538475, P47=0.47779376522243217, P48=0.5331081879785954, P49=0.5850115671200959, P50=0.6398117025798876, P51=0.6296681910732826, P52=0.0006064061026988223, P53=0.0011840857070741751, P54=0.0012042210439162395, P55=0.0008095834966085386, P56=0.0005217793545259447, P57=0.000289456480303081, P58=0.0001690693243446244, P59=0.00012303972816253217, P60=0.0001975689888670986, P61=0.00038111825530919187, P62=0.002982118145772112, P63=0.0036339576330100326, P64=0.0007330450684258736, P65=0.00031530707354727767, P66=0.0003124185701244153, P67=0.00039333274342470443, P68=0.005407608148438797, P69=0.019226809134955655, P70=0.018459368917950046, P71=0.027768449488096213, P72=0.0758306436941559, P73=0.25374165616195016, P74=0.8126767488976866, P75=0.6914670172638957, P76=0.7804435306313472, P77=1.6068508465233293, P78=2.075187252217658, P79=2.008923398861239, P80=1.5326057631346932, P81=1.3211664297851045, P82=1.2038509855986907, P83=2.040518421619049, P84=2.7963190546152545, P85=3.0524060282074816, P86=2.48339852707363, P87=3.7731051780746263, P88=4.3170936191200875, P89=3.1003482124373436, P90=2.9211611223947425, P91=2.5918880640916955, P92=0.9854105542183066, P93=0.27459969804911866, P94=0.27342507054613363, P95=0.32496283712318247, P96=1.32622957943014, P97=2.698465318729585, P98=5.126815445791351, P99=2.1961401507637137, P100=1.3926692038394157, P101=0.8373729982243349, P102=0.7985046231982874, P103=0.3596699177837935, P104=0.19736719555383603, P105=0.17426681889596532, P106=0.11695144179843853, P107=0.05086603498856876, P108=0.46775010443215564, P109=1.1743322759197223, P110=0.8517595083450576, P111=0.714659496619922, P112=0.2156817709984422, P113=0.29374163339253645, P114=0.23160224438048044, P115=0.06439356309311646, P116=0.025164568056046038, P117=0.15314571115371786, P118=0.23721738942286588, P119=0.8862680827990814, P120=1.7798850505399773, P121=0.20714688986510357, P122=0.1066599517586653, P123=0.04260670001328409, P124=0.038073045826997, P125=0.14762889527643278, P126=1.111117490257793, P127=0.4594636725039137, P128=0.16539426296629356, P129=0.151389901651374, P130=0.1247430096797328, P131=0.1895036787545907, P132=0.103840803622496, P133=0.2925771829657473, P134=0.10085559150836063, P135=0.04244436044570184, P136=0.12016786711977315, P137=0.07917141919182664, P138=0.059437188751393304, P139=0.08217346111054814, P140=0.08746079151466864, P141=0.054783104040713466, P142=0.03101675527190774, P143=0.061646915530890176, P144=0.07616894914632565, P145=0.04356994032208429, P146=0.029845421983539365, P147=0.07071723677601487, P148=0.050194236471621946, P149=0.022549742011011963, P150=0.014729177241124344, P151=0.0057991052360620654, P152=0.0027859604739426625, P153=0.0014469355920982761, P154=0.001477814776520361, P155=0.0026266543974372007, P156=0.0028159536674690834, P157=0.0037216900817765956, P158=0.0049229894262485235, P159=0.005121200593059949, P160=0.0066435777294898786, P161=0.007535017524760578, P162=0.006487428753173232, P163=0.007301391476718432, P164=0.008125907991779395, P165=0.010655626276582836, P166=0.010229705910551824, P167=0.011622102521798348, P168=0.012195480949705921, P169=0.013388514046549786, P170=0.0066374236707651925, P171=0.006479179420549883, P172=0.0068513108695014355, P173=0.007736345509407507, P174=0.009360729733034862, P175=0.010495299837943745, P176=0.012337594610170567, P177=0.009150392826755806, P178=0.003720943007698865, P179=0.00034631725855641853),Row( P0=-727.2005467142126, P1=54.89493494835548, P2=-10.870282921259925, P3=-6.0235648361479015, P4=-15.663014504510176, P5=-11.390903324193847, P6=-5.731322376744987, P7=-3.5563349144772745, P8=-11.128405779121161, P9=-7.910492362568481, P10=-5.3327911053224755, P11=-5.147818906893438, P12=-10.080982252224453, P13=-4.062970557388622, P14=-5.495766649757513, P15=-5.479453226323422, P16=-2.136694834379444, P17=-0.8525319645953964, P18=4.198382688246879, P19=5.7430201954584925, P20=4.516590900092174, P21=7.009611265331229, P22=4.180043547351145, P23=6.115008451658083, P24=3.327150725796538, P25=1.9935638453816957, P26=-0.9462468303570784, P27=-2.8242316344582648, P28=-2.831564233408411, P29=-0.8561463523243722, P30=0.42314792999862993, P31=2.193260698749285, P32=-0.15088019263464875, P33=-1.1867953533756734, P34=-3.6505828624306114, P35=0.15573987867287425, P36=0.6570951624557931, P37=0.47267928293397804, P38=-2.881968895516218, P39=-2.741012461664753, P40=0.4463489582678313, P41=0.4846938079394956, P42=0.46604425642579494, P43=0.4408720844167486, P44=0.4704060724062692, P45=0.5232757100318736, P46=0.5769640809614498, P47=0.5051088313823093, P48=0.45786058082850917, P49=0.45936307622518296, P50=0.49829862732008456, P51=0.4637410412161775, P52=1.646201078529789e-06, P53=2.7326560452613366e-06, P54=1.5818471974611618e-06, P55=2.6980950013885743e-06, P56=4.867311385779228e-06, P57=1.0708223407619935e-05, P58=8.663394020458173e-06, P59=1.1235643295410437e-05, P60=9.941649695706697e-05, P61=0.0012825375830375408, P62=0.020827638620187598, P63=0.030710763664654202, P64=0.006655517511782952, P65=0.01156367272282875, P66=0.01319721298476747, P67=0.006798510632541688, P68=0.0011722069542936156, P69=0.00016675898459031277, P70=0.00010323968506069265, P71=0.00010041960280288897, P72=0.0004845747934637032, P73=0.0022947519691196923, P74=0.02031198926218338, P75=0.020036344926298518, P76=0.009033548420251546, P77=0.004465610208452669, P78=0.003679817366454407, P79=0.012020849556095167, P80=0.015718657613209428, P81=0.004473147382161995, P82=0.0017186648110690214, P83=0.0008218681675923039, P84=0.0009064091134576687, P85=0.00255266709833278, P86=0.00729750098838036, P87=0.0070096009982494, P88=0.007254063860414982, P89=0.001778541340850812, P90=0.00041579205273223287, P91=0.0036131641816824515, P92=0.015083175432339824, P93=0.007234515992726362, P94=0.0066215866591893355, P95=0.006407494149939445, P96=0.0009065830797961557, P97=0.0003473022350300363, P98=0.0003658958178776918, P99=0.0004305929099538223, P100=0.0004542052101803266, P101=0.0024848359137398094, P102=0.0035460461900783864, P103=0.003379423533545138, P104=0.0011542024045658658, P105=7.997141961347283e-05, P106=0.00017479979373228232, P107=0.00029472736524797824, P108=0.0004580979931699706, P109=0.0006476355640210107, P110=0.0028058205299270784, P111=0.0006884777445644354, P112=0.00014554276207662947, P113=3.882089415758027e-05, P114=0.00016307083166837956, P115=0.0007212833588484866, P116=0.0008717347994609059, P117=6.239928853859709e-05, P118=8.629491165179695e-05, P119=0.00018039096709091505, P120=0.00011794600109693167, P121=6.0089721511003694e-05, P122=0.0004496414451351647, P123=0.00031471834625738563, P124=5.608637863327411e-05, P125=7.302168988828927e-05, P126=0.00018666585690286822, P127=6.864525765882049e-05, P128=5.404367131882702e-05, P129=0.00013230615688123647, P130=9.25544036754798e-05, P131=4.6999787443479734e-05, P132=0.00011637223940377289, P133=2.301209573865379e-05, P134=3.0058087291301895e-05, P135=3.0256644270616074e-05, P136=1.4200839546307206e-05, P137=1.4186713942112233e-05, P138=1.20523492254508e-05, P139=1.127349776878277e-05, P140=8.8517521241313e-06, P141=5.7106615633068485e-06, P142=7.3128992531369835e-06, P143=9.850944749466559e-06, P144=9.777429043211398e-06, P145=1.2374384408085594e-05, P146=1.2157073145523869e-05, P147=1.62892354118347e-05, P148=2.014384401960514e-05, P149=2.644844759514081e-05, P150=3.641824154004482e-05, P151=4.2658777997234063e-05, P152=4.712304361925263e-05, P153=1.4645876236959548e-05, P154=1.5179885632966153e-05, P155=1.1180870457126774e-05, P156=7.740266704904002e-06, P157=4.115300810720953e-06, P158=5.929274526787757e-06, P159=1.3183656775231045e-05, P160=2.9678675722584443e-05, P161=5.265626770476152e-05, P162=6.486349719615723e-05, P163=9.052593979926096e-05, P164=0.00010264903852454952, P165=0.00011175005925449701, P166=3.371838283226529e-05, P167=1.1649239577533406e-05, P168=2.092839346892614e-05, P169=1.999502359943471e-05, P170=3.267891854020195e-05, P171=8.223076330442011e-05, P172=6.397014762255898e-05, P173=4.3979143274255544e-05, P174=2.4833192963047753e-05, P175=1.671933025947303e-05, P176=1.667591646543885e-05, P177=1.867403182537188e-05, P178=8.990758803112526e-06, P179=9.71139285389809e-07),Row( P0=-430.20199536136477, P1=5.615958536047477, P2=-37.72732570804445, P3=-8.52849896140768, P4=-26.27759198114234, P5=-15.305501388523485, P6=-0.35399907103348255, P7=5.962505840294405, P8=11.297849098972687, P9=11.50905490988468, P10=9.7796973859366, P11=9.333320018465491, P12=-10.42582479307856, P13=-0.9941724531309355, P14=-0.0073693417036103746, P15=-7.694850351638694, P16=2.4000606132818896, P17=1.4058330510506056, P18=-0.11892903126548966, P19=-1.9461249098997193, P20=0.7869639822537117, P21=6.498708404981407, P22=3.24741493990206, P23=1.6870316145877309, P24=-2.620880138535682, P25=-2.2278197860388413, P26=-0.01866062165238636, P27=0.6104969556796829, P28=-3.146670806856763, P29=1.2251462349462667, P30=0.007895821121781202, P31=2.7023678649773926, P32=-2.4229392179971736, P33=-0.03290171389601898, P34=1.2565108513623573, P35=1.5136136584899518, P36=-2.5575749057844503, P37=-1.6786644968259226, P38=0.07844811448366532, P39=1.963393317385821, P40=0.6868494305159836, P41=0.6954627499281087, P42=0.6586936159653657, P43=0.5847573116947666, P44=0.5188691374207309, P45=0.426343061598344, P46=0.4126321445258161, P47=0.4893654934346225, P48=0.5478403729253121, P49=0.5990304343575461, P50=0.6407994668110393, P51=0.6728336280327121, P52=0.003576020488948033, P53=0.00426663658196975, P54=0.00283151943109645, P55=0.0012015671405384712, P56=0.0007201488087105038, P57=0.0005329497453989159, P58=0.00014555531419678592, P59=0.00012225792989639723, P60=0.00016043279402986954, P61=0.00017673079022194015, P62=0.00018202627249499115, P63=0.00014129510558863053, P64=9.904188251892694e-05, P65=0.00015088108133546674, P66=0.0001964795794905562, P67=0.00017337042907311414, P68=0.00020122861715895466, P69=0.00032743218042339856, P70=0.00047618946251981977, P71=0.002167715864060228, P72=0.008819910962497724, P73=0.016850936548074385, P74=0.06752231496097937, P75=0.24037523143888662, P76=0.46302218129770834, P77=0.7140061032200629, P78=1.1575706206224412, P79=1.5630147206300262, P80=1.1319490778404941, P81=1.1055748278170254, P82=1.118680036429841, P83=1.2932018888659687, P84=3.230486628873747, P85=9.034003925742962, P86=6.79040419288843, P87=4.171234276628, P88=6.669354157103582, P89=10.832082866570607, P90=10.672501475671927, P91=6.721126568961127, P92=0.5774558253469974, P93=0.010073938243951005, P94=0.041656046888269814, P95=0.7805765650437971, P96=2.8132273341991234, P97=2.9299767490392434, P98=2.1984453482547925, P99=1.5723963302309518, P100=2.1489032176106675, P101=0.8407271464875954, P102=0.5372574009426673, P103=0.328496244551463, P104=0.46765003407519307, P105=0.27671758210134434, P106=0.18304514154114082, P107=0.07045527321848231, P108=0.08461629068466009, P109=0.6576110574266831, P110=1.2803436932858379, P111=1.0279446199407856, P112=1.51702544166294, P113=0.3361249382020349, P114=0.06020240930522177, P115=0.035353931282825504, P116=0.03461920159086581, P117=0.88668321021353, P118=1.9242519584254663, P119=0.38041870028542973, P120=0.11505209806485256, P121=0.08243080715623592, P122=0.07506445355712282, P123=0.10453270328845471, P124=0.38794488318034204, P125=0.28165617101983453, P126=0.10887681526857299, P127=0.0827810442610319, P128=0.05506093659914374, P129=0.1597217075667496, P130=0.21848765241522466, P131=0.15539003064112394, P132=0.14340765888539422, P133=0.202519242479615, P134=0.03648764011829621, P135=0.0225752459558414, P136=0.012149622436355652, P137=0.011140561203097204, P138=0.01218414277980815, P139=0.017131475518606214, P140=0.023790325532802668, P141=0.008488096705776212, P142=0.008720765634954664, P143=0.012487867872590524, P144=0.01141225239708705, P145=0.022597985800577088, P146=0.02988044377636427, P147=0.014922004508836913, P148=0.012767715186553253, P149=0.015704285353553688, P150=0.014585084687275124, P151=0.005166090160259055, P152=0.0031694511840690454, P153=0.001670201366010564, P154=0.0013564460936636668, P155=0.00096089297534479, P156=0.0014919551169121695, P157=0.0016137736425151903, P158=0.001962969609258779, P159=0.0037539885008365634, P160=0.0026720728781104135, P161=0.0021986628201593906, P162=0.0018828240687043434, P163=0.0019825030820278928, P164=0.002260610260488455, P165=0.0017615898598531707, P166=0.001948396996195012, P167=0.0020232334158137074, P168=0.002073339441551208, P169=0.001995079072769208, P170=0.00213033772488245, P171=0.0013431681465429876, P172=0.0014951291815645132, P173=0.0022904435341088063, P174=0.0030364758355483456, P175=0.003780463445699544, P176=0.0037785647070015652, P177=0.002474497167708847, P178=0.0012737467495667553, P179=0.000204238124478132),Row( P0=-590.9345718916169, P1=46.74989089325818, P2=-6.310992588084667, P3=-10.872358575373497, P4=-22.43379965382859, P5=-16.228245641241617, P6=-8.543808558264649, P7=-5.396956401017846, P8=-8.340452307490168, P9=-10.044761724073693, P10=-12.434003080059938, P11=-0.6786424828881882, P12=-14.729399968731675, P13=-6.0821383613148425, P14=-5.810795615793359, P15=-8.311661962653316, P16=-1.903090835939879, P17=1.9943149578704704, P18=2.415034943183424, P19=6.115716892422984, P20=6.64341596144807, P21=5.316369945014385, P22=1.6253655320771538, P23=2.352999257272887, P24=0.351511290636618, P25=-1.0692593264523962, P26=-1.1323746226786497, P27=-1.9167178751366294, P28=-3.035649229207506, P29=-0.9007745048827956, P30=-1.0182023512943619, P31=0.7556030789999294, P32=0.20588801906565585, P33=-0.6956546428226493, P34=-2.9130092080100063, P35=0.0034882200475693207, P36=-0.23045423239660057, P37=1.533175800771119, P38=-2.3986971612032746, P39=-0.5735466254024614, P40=0.5824809979653343, P41=0.6095826347994028, P42=0.5880274003444167, P43=0.5518622775413122, P44=0.5409660194694769, P45=0.5752265466350713, P46=0.6469335857691833, P47=0.6340779987996492, P48=0.5826445543589613, P49=0.6311383491786571, P50=0.642118723316025, P51=0.5982392441782596, P52=3.672696734007248e-06, P53=7.016338768446686e-06, P54=1.3991821029387767e-05, P55=2.995860242298361e-05, P56=4.289606757609872e-05, P57=6.943455842177051e-05, P58=7.11036709050598e-05, P59=9.377901106940891e-05, P60=0.0004821929502454218, P61=0.004242445467980658, P62=0.046709499443934266, P63=0.06704575931069925, P64=0.04787243872573523, P65=0.17338237491856678, P66=0.1903860845229808, P67=0.07711812309110636, P68=0.019820321646684617, P69=0.004184050289324506, P70=0.0015528849073911805, P71=0.00072421283963146, P72=0.0016405792387991314, P73=0.010055690037242833, P74=0.05158313751889399, P75=0.06063177198080887, P76=0.04822882260887633, P77=0.07352996578796353, P78=0.1680860987927905, P79=0.31503624301539895, P80=0.24202463211247585, P81=0.12182378565707526, P82=0.06486192084119469, P83=0.04067270622018804, P84=0.021698045253460408, P85=0.014672322106492802, P86=0.015740216917260005, P87=0.024671475200889754, P88=0.039511579561150624, P89=0.06349692082552905, P90=0.0657837593733695, P91=0.0656889397963372, P92=0.04678469689428227, P93=0.0474279619710716, P94=0.056706376514307766, P95=0.03499551149947609, P96=0.007609638119743732, P97=0.0035319343182260783, P98=0.0058206363733290635, P99=0.005518966528393033, P100=0.012050410346416851, P101=0.053019373728787816, P102=0.1409770752388761, P103=0.046281904821066956, P104=0.0058538413835980295, P105=0.0016265707599174115, P106=0.0009492759951464996, P107=0.0015325846773974541, P108=0.005710027829091715, P109=0.005321318325153328, P110=0.00825353338120182, P111=0.0013419036575706776, P112=0.0005421153457162533, P113=0.0015324222510275623, P114=0.0020632025132086814, P115=0.0020790142384602348, P116=0.001835510014719449, P117=0.0014866205715203633, P118=0.0004761479759454318, P119=0.0008089228526644876, P120=0.0007793703223573908, P121=0.000609261377143792, P122=0.00020548007247302963, P123=0.00045924116588056974, P124=0.0009387054919483225, P125=0.0005872918126158507, P126=0.0004754612232569819, P127=0.0005981970786296678, P128=0.0007335569548772461, P129=0.0006511734836933854, P130=0.0007183923582091448, P131=0.0005888639060350864, P132=0.0005478549700345537, P133=0.0002722579115624626, P134=0.0001145387473961754, P135=0.0002873956901155097, P136=0.00021554093602168685, P137=0.0001920314850460774, P138=0.00022874154979208707, P139=0.00023540262131962553, P140=0.0003519772021827694, P141=0.0007257986463057515, P142=0.0006117190534257769, P143=0.00045523733841068743, P144=0.0007329142804035512, P145=0.001195950002354764, P146=0.0010271700419861828, P147=0.0010420364756591719, P148=0.001068365382078019, P149=0.0007810636479179938, P150=0.0005459551056170097, P151=0.000542039587214582, P152=0.00028102038155622156, P153=0.00024000440347265553, P154=0.0003385681905373069, P155=0.00025037367421807847, P156=0.00024991440410424064, P157=0.000704950798642959, P158=0.0009714391939163427, P159=0.0010924891058475335, P160=0.0022625900357333326, P161=0.003952307609871217, P162=0.002822483709367113, P163=0.002952538512704422, P164=0.00304326385541888, P165=0.003224686813423671, P166=0.003704349490666253, P167=0.00465002516875883, P168=0.0030096487367373475, P169=0.005721224077055783, P170=0.0068720772187435995, P171=0.006647357996743159, P172=0.004323352890957611, P173=0.004251898077009164, P174=0.0030086285127203993, P175=0.002788738548387625, P176=0.0019408664965547022, P177=0.0013761964796627079, P178=0.0005590631380127917, P179=4.935804654289945e-05),Row( P0=-602.3507236607915, P1=40.23435465821406, P2=-20.818869661062877, P3=-4.543141075626705, P4=-20.90045870233443, P5=-15.18997371011512, P6=-8.156080433009663, P7=-5.53795088666903, P8=-5.1418802885918735, P9=-8.686326069865489, P10=-5.623414693771663, P11=6.330468417313233, P12=-1.4500161669243217, P13=8.442713111017497, P14=13.448711482032262, P15=-2.304499914212157, P16=-2.947321842767283, P17=-0.9242575702316863, P18=-1.9148425553045425, P19=-0.45044861061305747, P20=1.5040548462344687, P21=1.8606598780851982, P22=-0.22900221504537513, P23=-0.33625210824533464, P24=-1.531752465303533, P25=-2.951642911786013, P26=-2.6424381288500682, P27=1.4268995598687537, P28=-0.4466349925178282, P29=1.1649790525280337, P30=-3.6821920642258674, P31=0.5973991698496167, P32=1.9864617555469568, P33=-0.321582868015373, P34=-0.7739874880477664, P35=-1.1086253745448935, P36=-0.004124017805701019, P37=0.7762103911208282, P38=-2.465840332783934, P39=0.442168189610891, P40=0.6063056066193658, P41=0.645311049371401, P42=0.5865284769044641, P43=0.5471525947666949, P44=0.5380076321249452, P45=0.4956889639472536, P46=0.5092090721010678, P47=0.49966394222626087, P48=0.5202957709594693, P49=0.5446042801239008, P50=0.566340076773892, P51=0.5846227408420999, P52=6.458494742967814e-05, P53=3.6417629917532856e-05, P54=4.419325703285721e-05, P55=5.8604026324794026e-05, P56=6.36687205780597e-05, P57=7.606520167103459e-05, P58=6.233745411829257e-05, P59=5.929847306212242e-05, P60=6.159863454729547e-05, P61=8.795837759454737e-05, P62=0.0006723501389718562, P63=0.00434195916304745, P64=0.015102222569491855, P65=0.02287607625986519, P66=0.023626681934713987, P67=0.03292482087216304, P68=0.06620515232307814, P69=0.14235812360931785, P70=0.1530312388770911, P71=0.12278934579598196, P72=0.12876520575433142, P73=0.1416026726504039, P74=0.33567517727256446, P75=0.3104648936560569, P76=0.12452167881572476, P77=0.051177872808014155, P78=0.0369711726528694, P79=0.017040980898444625, P80=0.004917016648407895, P81=0.0029767989311805464, P82=0.0025793295182755314, P83=0.003736329031896847, P84=0.021268600649026748, P85=0.10370337585968437, P86=0.12013177959723265, P87=0.08730839040052621, P88=0.06729184459718272, P89=0.06576248981196164, P90=0.10140008541530218, P91=0.06084103544613066, P92=0.01872463214121317, P93=0.018745591964077376, P94=0.07190447753530292, P95=0.13864137549137848, P96=0.029043056243955014, P97=0.10962913618615941, P98=0.21495288399757628, P99=0.08712233050138508, P100=0.023053745527966636, P101=0.004071307067954794, P102=0.0015082360782200268, P103=0.0013425236790911745, P104=0.0037795810509501024, P105=0.0161605683459123, P106=0.05748781285291059, P107=0.15805067206277143, P108=0.13515547072570097, P109=0.007992742338727609, P110=0.0008609080317277279, P111=0.0005013374705182233, P112=0.00046569761999711164, P113=0.005102705566283876, P114=0.02541667355503649, P115=0.03779747177722005, P116=0.009176350311930823, P117=0.0025194854370481487, P118=0.0006647273228215688, P119=0.008870784719923988, P120=0.013879946761459117, P121=0.0021131684503030795, P122=0.002170980866367544, P123=0.004080289465606096, P124=0.0215970422615619, P125=0.005426245582984756, P126=0.0016843843063901105, P127=0.002067861177283673, P128=0.00639208203174845, P129=0.002363253537568423, P130=0.000800191848773808, P131=0.0013460026570669714, P132=0.0013304374282435481, P133=0.0003216382750953247, P134=0.0003020671327621714, P135=0.0004694224405131037, P136=0.00014404572487890596, P137=0.000474731806432201, P138=0.0014011243194480684, P139=0.0010666388558592476, P140=0.0013776573754127166, P141=0.0013924197288242, P142=0.0005318980133456413, P143=0.0006591728837270207, P144=0.0003185823251455267, P145=0.0006207729046018568, P146=0.0005244822622989507, P147=0.0009272770657901646, P148=0.0009983982120659984, P149=0.0018978912806529489, P150=0.0005663837810982581, P151=0.00021814258332547527, P152=7.78247147560208e-05, P153=8.416527075527236e-05, P154=0.0001649798478148044, P155=0.00022249988134367735, P156=0.0002652687098797485, P157=0.00029159581547722027, P158=0.00020430798538300572, P159=0.000256286119196128, P160=0.00016699075415136836, P161=0.00014092848687186614, P162=0.00011890122548796835, P163=0.00018827384413881877, P164=0.00025307792540550856, P165=0.00024862047012626725, P166=0.00022436965931054036, P167=0.00018297146698568494, P168=0.00017810557752990814, P169=0.00015378293453564607, P170=0.0001515432972528262, P171=0.00013875818730656073, P172=0.0001564636641669957, P173=0.00015548706010044426, P174=0.0001537608377924799, P175=0.00017239531789505419, P176=9.714496688486353e-05, P177=8.78241891560113e-05, P178=4.1231439404435674e-05, P179=9.304674821200419e-06),Row( P0=-571.7157376276359, P1=31.98018092545825, P2=-18.037543238408812, P3=-9.388074343288928, P4=-21.43024473041684, P5=-18.600333222380947, P6=-9.53447925688428, P7=-4.110940756085903, P8=-7.727612438117269, P9=-11.202591678782957, P10=-6.628964258424821, P11=4.296334424584881, P12=-4.52709706694574, P13=6.309081836561126, P14=9.659444850896564, P15=-0.7241235328877667, P16=1.9612232489002606, P17=-0.9495755117593733, P18=-2.8188082965228065, P19=-0.048279576596331544, P20=0.1485208429713155, P21=2.536393759355184, P22=-0.9035419507592858, P23=-2.6894782058063305, P24=-1.3642279296441169, P25=-2.23932847384604, P26=-1.205797830699515, P27=-0.14455814960956767, P28=-0.29499219618759664, P29=2.0195733608166364, P30=-0.8411775459555837, P31=3.2621086792357312, P32=-0.41003425169124563, P33=0.4158374211844373, P34=-1.4830479241444077, P35=-1.6665158692673052, P36=-2.0842014513135823, P37=-1.7664778655962465, P38=-1.6984015105041008, P39=1.017329808397533, P40=0.5342202766367217, P41=0.5328728914857229, P42=0.501959401123779, P43=0.4759449066856222, P44=0.4582970947260018, P45=0.41142053701913367, P46=0.42110947501503715, P47=0.4144724866579791, P48=0.42703554788030734, P49=0.4535599375815678, P50=0.5121121205807573, P51=0.5311298382008152, P52=6.9207520083840035e-06, P53=1.4565082366695037e-05, P54=1.8067648159206203e-05, P55=1.768397053809094e-05, P56=2.019445589215811e-05, P57=3.172266762785266e-05, P58=2.977169763410022e-05, P59=3.097267129425335e-05, P60=0.00013001933910236353, P61=0.0006511387719842149, P62=0.0026037683187177547, P63=0.010206289985428661, P64=0.02746147929001097, P65=0.06851142881198241, P66=0.1144291397823458, P67=0.11764438969084089, P68=0.22914005227093465, P69=0.3496513050076357, P70=0.32533274690175656, P71=0.16659590251431486, P72=0.13113598462011916, P73=0.06245676948099592, P74=0.08167157112827754, P75=0.12369043143137069, P76=0.06618542108210858, P77=0.05338089035629475, P78=0.03506596469531173, P79=0.02329449703081603, P80=0.02090084069789819, P81=0.022154730734230757, P82=0.028506438056909705, P83=0.060908323519393445, P84=0.09600435389355261, P85=0.1463944343812533, P86=0.1336712527984305, P87=0.24749591052870745, P88=0.9260886787493511, P89=0.3638053572499509, P90=0.10785265461224357, P91=0.11908841293959802, P92=0.040621979602344295, P93=0.019038838526913685, P94=0.02518309876406752, P95=0.027448630147380193, P96=0.010045956413570194, P97=0.009048229209479711, P98=0.007849183587378157, P99=0.005793663669469696, P100=0.024213400749747344, P101=0.022514541298649066, P102=0.022737123887174693, P103=0.011729237071454497, P104=0.003064936918321743, P105=0.0026494891963676414, P106=0.002130353174823158, P107=0.000979918465630161, P108=0.001095479237594328, P109=0.0027316879588119877, P110=0.0021498258910916785, P111=0.0013089136082077822, P112=0.0012199965761003862, P113=0.0014272688110377927, P114=0.001720571692727709, P115=0.002527146902859511, P116=0.0027207328631169707, P117=0.002493923490841389, P118=0.00258625977916476, P119=0.0020393045482144075, P120=0.0018887267057601824, P121=0.0009782823803959693, P122=0.0010434501752086152, P123=0.002175326496576923, P124=0.002365159981627209, P125=0.0020476611149016603, P126=0.001633928406011535, P127=0.0010875417430207166, P128=0.001130658074866033, P129=0.001238842314671944, P130=0.0010487174866225606, P131=0.00029365454343622296, P132=0.00016375662216687264, P133=0.0009784492174287427, P134=0.0009502585050298892, P135=0.00022362334871810178, P136=0.0004776158400272892, P137=0.0013346139603861832, P138=0.0008914407588186984, P139=0.0006321565137799279, P140=0.0005283471247297034, P141=0.00020643684019588398, P142=0.00020496988082338723, P143=0.00040096529354544824, P144=0.0003115271995420828, P145=0.0004107134337476658, P146=0.00031655272281549106, P147=0.0002526430411425983, P148=0.00021726385818256983, P149=0.00029307172985110756, P150=0.0004935237954506766, P151=0.000670809753025024, P152=0.00041339430563825606, P153=0.00014066997386269289, P154=7.355953492120579e-05, P155=6.03222163159959e-05, P156=5.7215110620487766e-05, P157=0.00011378144709047173, P158=0.00018290039652595752, P159=0.000281061146289512, P160=0.0003780761672099074, P161=0.00046673558257913486, P162=0.0003708402550693479, P163=0.00043487842472045453, P164=0.0005040502924213715, P165=0.0005618387332672949, P166=0.0004762199454123887, P167=0.000799213993608846, P168=0.0014392788278057768, P169=0.0014521475412250387, P170=0.0025139817529671424, P171=0.0018542536845271142, P172=0.001107757905713777, P173=0.000830546092677851, P174=0.0011865340603581232, P175=0.0010549947879361925, P176=0.0009329018102932728, P177=0.0007899705170095441, P178=0.0004583462279991811, P179=4.7504482843490435e-05),Row( P0=-593.0153158634955, P1=36.56325997992817, P2=-15.888083923062878, P3=0.20462249123008525, P4=-16.719673978808736, P5=-10.011666217318677, P6=-4.696594964638857, P7=-2.495713334340829, P8=-6.947029866735905, P9=-6.825322449379142, P10=-2.3937428875888465, P11=1.7304709410247348, P12=-3.426061928735522, P13=4.996192510300881, P14=3.4236884988516465, P15=-0.5599787452640012, P16=1.9231159554577426, P17=0.15704128498533018, P18=2.4277824527739753, P19=0.3936290805720376, P20=-1.1948966035184085, P21=-1.2058255661468513, P22=-2.005501383849272, P23=3.1042520074011444, P24=3.6937714508949937, P25=-0.4363336674950754, P26=-2.419445321139093, P27=-1.7664356112346782, P28=-0.8564011582210773, P29=0.8163261058729065, P30=-1.0610530181857325, P31=0.9229801704421832, P32=0.40877314465098963, P33=1.07704783125328, P34=-2.6359892601993558, P35=0.11092925640258348, P36=-0.3652360781384796, P37=-0.2691085303871344, P38=-2.638770946554241, P39=-1.5167445427720767, P40=0.644363084228326, P41=0.7069543473493501, P42=0.7228538696198886, P43=0.6452149075355628, P44=0.6314774067975855, P45=0.6522606725600675, P46=0.6275353267393101, P47=0.6205875196486592, P48=0.5384780873288023, P49=0.5291109146307085, P50=0.5568328412905342, P51=0.6147937657944982, P52=0.00013297631157460033, P53=0.00011762413870101654, P54=0.00013495950656930853, P55=0.00028145073242239615, P56=0.00045242898290094407, P57=0.0004498062431243738, P58=0.0002344015088243594, P59=0.0001357916867337646, P60=0.000567562040920623, P61=0.003988876120012948, P62=0.006617848701887179, P63=0.004886001583387212, P64=0.005339662029974359, P65=0.047621880938799856, P66=0.20307894649216796, P67=0.1537703699459389, P68=0.06108727023552745, P69=0.0826707422462788, P70=0.04836703147414018, P71=0.02447869803578413, P72=0.069375387319857, P73=0.06919157385614633, P74=0.1012435310640362, P75=0.09794911846738003, P76=0.023547808310440316, P77=0.006001751947387082, P78=0.0054761562449933835, P79=0.007486010169288298, P80=0.01837495758039859, P81=0.04158498878601228, P82=0.05172345351991095, P83=0.0801187266097402, P84=0.1909357522459627, P85=0.3561331769701038, P86=0.24046339172651685, P87=0.2184652816524196, P88=0.15142189401932593, P89=0.0339713161789442, P90=0.016828221675085558, P91=0.031178832450475074, P92=0.033255921418596915, P93=0.02316890515271826, P94=0.1145683760821352, P95=0.17585278313711508, P96=0.04054298233009127, P97=0.01667094498983063, P98=0.014834671840972386, P99=0.005262388987301798, P100=0.002640379693801052, P101=0.006264583474596842, P102=0.11022587615841171, P103=0.07049281445501972, P104=0.0046242379768672, P105=0.0034680285467441475, P106=0.01655476460545406, P107=0.07245162700778618, P108=0.058683949423840136, P109=0.00881824354269109, P110=0.00240098356293158, P111=0.0009048558324518463, P112=0.0007158635421749993, P113=0.001826345992074607, P114=0.0036122219820325574, P115=0.007432052505263971, P116=0.014489474970368598, P117=0.014226658419289377, P118=0.0018029508440102532, P119=0.0035918249033314695, P120=0.004294773247334731, P121=0.001551545221670413, P122=0.0021547940830027846, P123=0.00751511509606358, P124=0.010514505674223434, P125=0.0026596023493482856, P126=0.001865107944492047, P127=0.0039726669705171195, P128=0.009491294587494389, P129=0.0026742863982027643, P130=0.0013700019504789324, P131=0.0027380503283016113, P132=0.0012704686993613725, P133=0.000716610521021753, P134=0.0015584129580403989, P135=0.0012206739418796617, P136=0.0006511739815055572, P137=0.0003730682674568826, P138=0.0016591753359912818, P139=0.001872733123452891, P140=0.000476843776526116, P141=0.0007611092220150969, P142=0.0005044631886980435, P143=0.0002666682434649242, P144=0.0003125501102787082, P145=0.00035832126472804584, P146=0.0003808558241726501, P147=0.0007742612369036688, P148=0.0014043619611194848, P149=0.0011146626134592448, P150=0.0007716516667899259, P151=0.000315721222587979, P152=0.00036269065205536325, P153=0.00019142739874407095, P154=8.069648999772505e-05, P155=0.00021016400378317095, P156=0.00037386157228833577, P157=0.000218867254861258, P158=0.0006146181935069768, P159=0.0006838870732295069, P160=0.0002366074173496438, P161=0.00013839283760593937, P162=0.0003380701677526247, P163=0.0002989410039209937, P164=0.00017603571774410493, P165=0.0001940072255442943, P166=0.00018800641236042898, P167=0.00025524191354786384, P168=0.0003108033831139823, P169=0.00034883030186832516, P170=0.00030775230024814254, P171=0.00017091014146533966, P172=0.00010751503714674192, P173=7.343090926846336e-05, P174=0.00011290994662800153, P175=0.00012828825337996518, P176=0.00019851330761075755, P177=0.00024877303380748083, P178=0.00010323556175927085, P179=1.1179795738878545e-05),Row( P0=-605.0124047048838, P1=37.67898555814208, P2=-16.4993676678117, P3=-5.242911146991051, P4=-18.59554168627664, P5=-15.92293599470544, P6=-5.90440933774216, P7=-4.077379655769083, P8=-9.253255501161664, P9=-9.623247546351253, P10=-8.79834865567462, P11=-5.3755353042734555, P12=-11.425589175826167, P13=-4.458110469395446, P14=-2.2990090936939422, P15=-3.0924480910957226, P16=0.8342664373951014, P17=0.817806423398026, P18=0.3978158994532963, P19=1.3173167363902234, P20=1.6425344189340259, P21=3.707235146257449, P22=5.134433286645691, P23=4.30304181908455, P24=0.042143241845809844, P25=-2.6164484792484157, P26=-2.4608223520576384, P27=-0.5880528127744191, P28=1.7324976679987447, P29=2.9030537651828014, P30=-2.5247401610810276, P31=-1.1340085984244397, P32=-1.187892076060337, P33=2.9919085620093964, P34=0.39278181676499857, P35=-0.39796357236715, P36=-4.207313669883297, P37=-1.6426071974787622, P38=-1.1801743860654748, P39=0.9766540175791548, P40=0.603191786605483, P41=0.6457212363001481, P42=0.6765340141444746, P43=0.6513718429999248, P44=0.6477254567677426, P45=0.594395628917606, P46=0.6290065388193125, P47=0.5468447955262403, P48=0.5894834592168003, P49=0.5762723567709085, P50=0.5045105931804464, P51=0.5269814842578922, P52=1.0034084341268414e-06, P53=1.8315358819460312e-06, P54=4.189668476991172e-06, P55=1.0576484777542484e-05, P56=1.5291117933750293e-05, P57=1.7427369378004963e-05, P58=1.2524881406233865e-05, P59=2.2573610361246698e-05, P60=0.000571114262503268, P61=0.00502779132997455, P62=0.017483958124906968, P63=0.04474481884102773, P64=0.04177588304574356, P65=0.032394963212822724, P66=0.06055076307530437, P67=0.08609686022508498, P68=0.05597299650314844, P69=0.06534005103656548, P70=0.025388916983235737, P71=0.004499621388210114, P72=0.0032512274215126874, P73=0.0028221972956648904, P74=0.010105698964988481, P75=0.02798156928261821, P76=0.04844524588204077, P77=0.09581266083224678, P78=0.06307205162319748, P79=0.030393036826847575, P80=0.022908946021802164, P81=0.022117007129457668, P82=0.034187857918446515, P83=0.04561613879750138, P84=0.09153727825420409, P85=0.2740017579817527, P86=0.30382251395980975, P87=0.13508117368454994, P88=0.07502956885967167, P89=0.04993807165107497, P90=0.02168751533231461, P91=0.014563616408463247, P92=0.010566688781153908, P93=0.003525824507032246, P94=0.011978734795291178, P95=0.07718217511597314, P96=0.03485997517693337, P97=0.0629081620967477, P98=0.14420752280141116, P99=0.05202954827438438, P100=0.004233247847280619, P101=0.0009090372708904285, P102=0.0003114379183746243, P103=0.008752129249328387, P104=0.038471302301212, P105=0.029016875061264706, P106=0.01730094404534273, P107=0.00711150185526408, P108=0.0018273977607828002, P109=0.000501200132091024, P110=0.005134014877491418, P111=0.0049351620480109675, P112=0.0022249889366962156, P113=0.0017785120625921395, P114=0.005146660343064937, P115=0.0069867476944989225, P116=0.010031180779481163, P117=0.0022246099435779976, P118=0.0014240942114708264, P119=0.008630338154567546, P120=0.003117890403436763, P121=0.002574658030667253, P122=0.0043170297786678516, P123=0.008235735464344739, P124=0.0037261267106685906, P125=0.002604477749143804, P126=0.0024328529394489636, P127=0.0015203807786810075, P128=0.0009522597926610865, P129=0.0027259469526676725, P130=0.0011161070886985432, P131=0.0017682038127151576, P132=0.0038000799851265065, P133=0.0025124552226654813, P134=0.0016358735746710466, P135=0.00136376435766134, P136=0.0003517942885792991, P137=0.00035724492324995223, P138=0.0005373431171084074, P139=0.0005234692551511954, P140=0.0009523379373922768, P141=0.000655461661114358, P142=0.0007750553734631773, P143=0.001693814504189396, P144=0.0005294493767658796, P145=0.000784510645131156, P146=0.0004242317050624648, P147=0.00045873770170148985, P148=0.0005635435954306298, P149=0.0004767263278335078, P150=0.00028804179321262146, P151=0.0001679516838859132, P152=9.829938725450399e-05, P153=7.434526732754471e-05, P154=4.962515309403978e-05, P155=5.7862134747027666e-05, P156=6.303785412207965e-05, P157=0.00011075568823220722, P158=0.00023490810269985386, P159=0.0001665008061428735, P160=0.0002742188408507129, P161=0.0002984379005400634, P162=0.00024050606382432425, P163=0.0001986119724230588, P164=0.00028979825714858836, P165=0.0003565394112866373, P166=0.00029701470079127794, P167=0.00028404856866842747, P168=0.00045600270916934413, P169=0.00029524983918042034, P170=0.0003884046139084099, P171=0.0002779445600831153, P172=0.00019088481334559682, P173=0.0003574456531200121, P174=0.0005435877617386371, P175=0.0005017203172775412, P176=0.0005152061614400181, P177=0.0003208561827945548, P178=0.00023288772316407654, P179=1.3175112210319843e-05),Row( P0=-490.6104631011374, P1=24.001075999637315, P2=-25.150078512407852, P3=-1.2045161188650804, P4=-19.194077338966444, P5=-11.765266510099398, P6=-2.69769116393732, P7=-5.773198489036961, P8=-5.874036339116369, P9=-3.5897876970194504, P10=-0.6869435065771056, P11=9.930271986090077, P12=2.138413303012035, P13=9.389286578989331, P14=6.428719185254163, P15=-1.438752755065959, P16=-0.5375637388352291, P17=-0.9622210521278165, P18=-0.5172763076851192, P19=-2.7451472970478834, P20=0.1716225864635258, P21=1.5039671102642758, P22=1.7085651426284156, P23=1.9012082831139687, P24=0.12231730957430799, P25=-1.9351659896237354, P26=-0.49088680090249437, P27=1.9918291703072288, P28=-2.264696282834202, P29=1.6191749201301262, P30=-2.368407582990772, P31=2.3668598694633665, P32=0.45651777687467354, P33=-0.40300867484343866, P34=-0.6224515027032012, P35=0.41278187498321434, P36=-1.6791350932852438, P37=-0.9805549524029661, P38=-0.7421016174129554, P39=0.6002136796571925, P40=0.6586905940102514, P41=0.7229671487920457, P42=0.6955575304728747, P43=0.65565111650989, P44=0.6416249635221125, P45=0.6575456845862597, P46=0.6277442538725384, P47=0.6410155207043429, P48=0.6320740555698562, P49=0.5843798340794134, P50=0.5776406225899707, P51=0.607316984381596, P52=0.004400989358996737, P53=0.003087236373237818, P54=0.0009276594842786618, P55=0.0005112343515029554, P56=0.0002746736325637439, P57=0.00027698081485615447, P58=0.00022754986376623563, P59=0.00016186186695143215, P60=0.0001988309287255147, P61=0.0019586890040933723, P62=0.012569034584173414, P63=0.010767882308704256, P64=0.007059996309384854, P65=0.016234352265453514, P66=0.05153982181240837, P67=0.07661562717015753, P68=0.1255840327922446, P69=0.2809956055714851, P70=0.395836502453342, P71=0.4277749188566744, P72=0.5296831998760104, P73=0.5872014930300791, P74=0.4476329310367418, P75=0.5166964057964071, P76=0.8365642495380335, P77=1.455702499312515, P78=0.8735160365476033, P79=0.30361821017913704, P80=0.08037768293348241, P81=0.03823710405237173, P82=0.05645448110521356, P83=0.11748038936222997, P84=0.18287962757161294, P85=0.4161871307562986, P86=0.6190169749049956, P87=0.3442625795438498, P88=0.2787445634247059, P89=0.26848107372569774, P90=0.22152082935706507, P91=0.16553493910505185, P92=0.18422422677041717, P93=0.2399469986449676, P94=0.13887404399860553, P95=0.10072260736083659, P96=0.30079287392716414, P97=0.582865692502067, P98=0.573111127655527, P99=0.24259570399703956, P100=0.273770489496969, P101=0.2400117060138685, P102=0.11254801018920811, P103=0.022002213300181428, P104=0.024941841274577, P105=0.04408390588819908, P106=0.11235492982808787, P107=0.23013130061619239, P108=0.32046664051650503, P109=2.8288050201275885, P110=1.1108819127287786, P111=0.027256327590918828, P112=0.005405082677277092, P113=0.011736465039201189, P114=0.05527827481148682, P115=0.08741317919298221, P116=0.09638399406302009, P117=0.11294341186802419, P118=0.07435761779336372, P119=0.024947690566885034, P120=0.03475775566814375, P121=0.04693392398273018, P122=0.04951461831966192, P123=0.019168663691290264, P124=0.058727029781978024, P125=0.030932773570346274, P126=0.02167994536920501, P127=0.013971279064211629, P128=0.02113990464625604, P129=0.02759413569758043, P130=0.007894770848116628, P131=0.06932946837804023, P132=0.05604201874092985, P133=0.010528619743999627, P134=0.005906816035662447, P135=0.01180052360314973, P136=0.030542355848981503, P137=0.012569693972886578, P138=0.017098067388450003, P139=0.012678042436110837, P140=0.037305033587540476, P141=0.019494458400087403, P142=0.007037206629711652, P143=0.0075864124559145655, P144=0.007608506498279054, P145=0.0037667901621832606, P146=0.0042687178807410435, P147=0.009821655019820835, P148=0.011529598870054681, P149=0.008498587101570554, P150=0.006747796376682307, P151=0.0015360291328784658, P152=0.0013542817417823661, P153=0.0010929547501924012, P154=0.0008088761807341344, P155=0.001495766607338561, P156=0.0033886693517911496, P157=0.00466247208687997, P158=0.0033270479247682946, P159=0.002974812527237268, P160=0.0015050960249358004, P161=0.001212122467224587, P162=0.0010980695709996025, P163=0.0011950896004450583, P164=0.0010602690269608556, P165=0.0008918073842507118, P166=0.001313106473641913, P167=0.0010518011019596248, P168=0.0006837544410998413, P169=0.0006294279758167759, P170=0.0010066014184821723, P171=0.0005666156867223414, P172=0.0003996912906231821, P173=0.0007041779089230382, P174=0.001530535273068699, P175=0.0019225376502218085, P176=0.0023938853817494854, P177=0.0014708438754297201, P178=0.00048634645278887727, P179=7.100226791176362e-05),Row( P0=-607.2513556963269, P1=47.071816282740144, P2=-10.417647216982166, P3=-6.370262168689848, P4=-17.109459830021432, P5=-12.569955647221853, P6=-5.437256283859091, P7=-2.048056373333473, P8=-6.715727299625347, P9=-7.166619679967775, P10=-7.08428667289581, P11=-3.4042756673777737, P12=-9.01629434643438, P13=-2.928730551777613, P14=-2.1561972942110503, P15=-1.6035790256995475, P16=1.058773663958709, P17=-0.06321121331386982, P18=2.316672618220273, P19=1.9759764443129555, P20=4.471100812926323, P21=5.421013674886893, P22=2.2240137910971685, P23=2.706987503159889, P24=-3.1909781998758415, P25=-2.2955244674709165, P26=-2.943983234259098, P27=-0.764605060812519, P28=-0.06794102261850397, P29=0.2106414081275106, P30=-2.2935519578457972, P31=-0.8038323484830285, P32=-0.22848400234779526, P33=-0.4276473576760335, P34=-2.143453390898275, P35=-2.6728656207536456, P36=-0.8567714264665908, P37=-0.6597336198244191, P38=-1.423087536762945, P39=-0.620738866852455, P40=0.5491190514211124, P41=0.5989881150069184, P42=0.6430495110930508, P43=0.603074526679369, P44=0.5578077817209401, P45=0.5314099516553952, P46=0.5071847706621609, P47=0.5227084306621971, P48=0.5314897019153805, P49=0.584163473931615, P50=0.5798991005488748, P51=0.5491711454867699, P52=4.74063943812809e-06, P53=1.8564933506817295e-05, P54=4.567337467649156e-05, P55=6.562468114121687e-05, P56=9.263502328525842e-05, P57=0.00013415246752457291, P58=0.00011696129200814144, P59=0.00017809355547925045, P60=0.0009511548249016797, P61=0.003944322508316466, P62=0.012487057954744186, P63=0.021897592393640575, P64=0.036828929643120345, P65=0.06147331002521095, P66=0.06837654961752929, P67=0.07120684737709454, P68=0.0824275085483517, P69=0.12028653010192396, P70=0.10534516027887689, P71=0.024419130536301577, P72=0.004108813446966792, P73=0.005972433689709493, P74=0.013327943592450054, P75=0.017132161021424994, P76=0.023027850411816694, P77=0.033872483467439825, P78=0.049069685272627304, P79=0.03893809314548926, P80=0.016865077664941885, P81=0.010985464153181473, P82=0.0095568296825994, P83=0.012629038450445851, P84=0.038440087636393915, P85=0.18160478036438785, P86=0.196592391678628, P87=0.06831935532649305, P88=0.03324185445285068, P89=0.04388413289436755, P90=0.061056243918815915, P91=0.042617725911279866, P92=0.026392168042035927, P93=0.01488627140017728, P94=0.01513423902572842, P95=0.015460297998956946, P96=0.01728213607316853, P97=0.041555670595837785, P98=0.07194975670996485, P99=0.0877309335854714, P100=0.06401644550272577, P101=0.03319875313761783, P102=0.017322708340504644, P103=0.010442045328151647, P104=0.01102682717417352, P105=0.012461318377741617, P106=0.021812068352942488, P107=0.015012281754866895, P108=0.006058739949192749, P109=0.005559131598724171, P110=0.0069865756126293015, P111=0.006105308076851059, P112=0.0023015083851519553, P113=0.0040981942850028385, P114=0.008391925122574527, P115=0.019934201077679205, P116=0.006902205128847367, P117=0.0014744831079875178, P118=0.0015308784191368082, P119=0.0012560029587920325, P120=0.0004814288139242349, P121=0.0013855789706807258, P122=0.002189145351276224, P123=0.0005118959128584696, P124=0.001616249306165131, P125=0.002857433911765877, P126=0.0006161233578458832, P127=0.0023759995318267018, P128=0.002145143862649182, P129=0.0004164886400912708, P130=0.0015100825105795357, P131=0.001234422059505555, P132=0.0010466564130803264, P133=0.0021881691779831753, P134=0.00030034385557478366, P135=0.0005802236718902022, P136=0.0005350342530053805, P137=0.0007927306723733791, P138=0.0011205421286000147, P139=0.0008737906003610467, P140=0.0006795919008863514, P141=0.000410496090291958, P142=0.00020262419447079056, P143=0.00026452436648668836, P144=0.000164121623173425, P145=0.00018875715352770708, P146=0.00018952547981817086, P147=0.0002125721385422374, P148=0.00030218269560420186, P149=0.0006091618612648696, P150=0.0006493234167614696, P151=0.000641853228539856, P152=0.0004210786559675671, P153=0.0003958313829435315, P154=0.0005124072702026613, P155=0.00044993468655834683, P156=0.0007936058389968316, P157=0.000959064823418903, P158=0.0008006765466519741, P159=0.0005314701123406694, P160=0.0005743066096727586, P161=0.0007685033906508495, P162=0.0007330149773023078, P163=0.0010724906090922128, P164=0.0011034407081165457, P165=0.0017622742595836726, P166=0.0025473223106169257, P167=0.0018381618074967546, P168=0.0007267072334243075, P169=0.0015743453328261653, P170=0.0020829701335900513, P171=0.0024403068253632376, P172=0.003095944666240444, P173=0.002622919609091276, P174=0.001597016139570985, P175=0.0009522922228007183, P176=0.0010289644093990794, P177=0.0006131324835611308, P178=0.0003254558533717942, P179=5.185543853997084e-05),Row( P0=-597.2201228921655, P1=47.48818282606361, P2=-13.824449138779473, P3=-5.806775466684806, P4=-20.601560784852488, P5=-8.554214612292396, P6=-6.805647662629602, P7=-3.5424567891849734, P8=-3.31293860545978, P9=-10.619849261628072, P10=-9.808981889411815, P11=0.7062868988586825, P12=-9.613842428560554, P13=-3.747557250125102, P14=0.9478047202383182, P15=-3.5999987618034015, P16=1.1118163144991855, P17=4.49475347345836, P18=2.6486100809607405, P19=4.335632186681787, P20=2.6012580921850295, P21=1.093994315013407, P22=1.203296744781155, P23=1.2658981235039009, P24=0.9720827706506295, P25=-1.8405189428765503, P26=-1.5769172556492426, P27=-1.7948340277517143, P28=-2.381435020184764, P29=1.1628976666326865, P30=-1.424816993978923, P31=1.8463351775044243, P32=-0.002699684303602779, P33=2.0137232477139433, P34=0.1682472551056904, P35=0.955437823153899, P36=-1.3252410006029165, P37=0.4417670216639945, P38=-1.0913569401065275, P39=0.5439584094215801, P40=0.5959362548527303, P41=0.5108433522386528, P42=0.4568523986333971, P43=0.47772964642058074, P44=0.4959792939685153, P45=0.4831541624429268, P46=0.5174810470609904, P47=0.5153422115663834, P48=0.5262977110467942, P49=0.5323356962933918, P50=0.553944951111513, P51=0.5975344178847067, P52=9.54302863499565e-05, P53=8.42095285249994e-05, P54=3.091934819349145e-05, P55=3.2213185615755464e-05, P56=5.086871712541731e-05, P57=9.180737077479303e-05, P58=7.136087170201833e-05, P59=8.707068211246776e-05, P60=0.0007603632924262735, P61=0.0037179648078127163, P62=0.007848451839705861, P63=0.015500226147421052, P64=0.02874266376977393, P65=0.04848370872755756, P66=0.06679516687070046, P67=0.09558095077193891, P68=0.045522004423815175, P69=0.006707539202416448, P70=0.001828554984833271, P71=0.0028402354777499824, P72=0.010578219460772972, P73=0.010970229453502617, P74=0.006291548617077202, P75=0.007762217797864344, P76=0.009811246367864618, P77=0.019776946364392724, P78=0.04013990027976316, P79=0.04136513834096588, P80=0.05981620038073776, P81=0.0433872904761199, P82=0.09334474676103256, P83=0.13613174475117296, P84=0.11304834089855503, P85=0.07501444246813321, P86=0.015382493346017424, P87=0.00732111090839184, P88=0.00894406954919277, P89=0.014166194130529507, P90=0.037084458666965504, P91=0.045894681158086315, P92=0.03985643800624738, P93=0.024643850810876906, P94=0.021938523922019067, P95=0.041381979635952776, P96=0.014737290693598032, P97=0.006013173017562147, P98=0.028363440554065276, P99=0.037537275594783225, P100=0.015641273743192498, P101=0.011159440204520397, P102=0.01320134544995844, P103=0.00665230538944797, P104=0.017787206464872404, P105=0.08064285948149234, P106=0.07949165455394254, P107=0.021125373142240642, P108=0.0036676094978205943, P109=0.0022513876241302064, P110=0.00743040763599295, P111=0.009586786303565478, P112=0.005278599857432833, P113=0.007246013595309458, P114=0.006119609913415246, P115=0.00594234869085262, P116=0.003304689932573011, P117=0.003648771729206378, P118=0.0026986531957891506, P119=0.0013081439620047032, P120=0.0018189145324837626, P121=0.0017243109651045951, P122=0.0015682563974217722, P123=0.0013407370107508482, P124=0.006383105551792129, P125=0.0032022121579568727, P126=0.0007789473456394053, P127=0.001050104448277302, P128=0.0017306469090524672, P129=0.0007912010808934423, P130=0.0005752211358712908, P131=0.0008937404636849516, P132=0.0008745139847892121, P133=0.0004734163252896542, P134=0.0004353130319828381, P135=0.0002539984436120925, P136=0.0002151233186361958, P137=0.00022495111860971497, P138=0.0003477873231793992, P139=0.0008255076694375032, P140=0.0014579042621066828, P141=0.0013042306208370655, P142=0.001149598262733053, P143=0.0012005520906013766, P144=0.000724985418905754, P145=0.0009413015219360835, P146=0.0010308043407881366, P147=0.0009742038612038273, P148=0.0007346759044137012, P149=0.0009172713951056015, P150=0.0007318925057540235, P151=0.000486951088174022, P152=0.00033268163830014314, P153=0.0002846364876949012, P154=0.00025943628429488623, P155=0.00022019482685979092, P156=0.0001679911930090581, P157=0.00019388727098718862, P158=0.00024828429057473407, P159=0.0002796225974674793, P160=0.00044847397927536535, P161=0.00041888439238640485, P162=0.00030699693070483257, P163=0.0002509483591145498, P164=0.00017181115892139624, P165=0.00020330752033887727, P166=0.0001671265423348965, P167=0.00018147444479419715, P168=0.0002048240633961093, P169=0.00018626541151976815, P170=0.0001380265940214426, P171=0.00011745816294568766, P172=8.37113982442131e-05, P173=0.0002210667083865693, P174=0.00020138969817693977, P175=0.0001129157763373762, P176=0.00010332467312093328, P177=9.857098258596126e-05, P178=6.194198072212494e-05, P179=7.262579975346901e-06),Row( P0=-675.3413987285916, P1=45.081380474398095, P2=-5.390665055049019, P3=-6.225071201818692, P4=-18.867665946511917, P5=-14.159834157202624, P6=-5.627689331898632, P7=-4.269867447743067, P8=-10.61873141293701, P9=-10.250137954854756, P10=-6.8261285446411994, P11=-6.34211170698598, P12=-11.600769432828866, P13=-5.488541738399194, P14=-7.249167519202187, P15=-6.9135054692348, P16=-2.7706752732057196, P17=-1.3526622699016448, P18=1.149090725327176, P19=3.588762832214043, P20=6.228779194191293, P21=6.47384839331006, P22=1.3247198133076294, P23=2.2144684829347576, P24=-1.889792417701539, P25=-1.1562137385564586, P26=0.1823240580092127, P27=-0.8983744711109345, P28=-0.8014596423362096, P29=0.8521203682003945, P30=-1.356733957744869, P31=1.1817490234265569, P32=-1.4487440309317068, P33=-0.3991416178998633, P34=-2.8992733280642886, P35=-0.03399354819603843, P36=-0.3191388833513382, P37=1.457049590870005, P38=-1.5387456725765303, P39=-0.8043228495673014, P40=0.5101398402180026, P41=0.5534394331278217, P42=0.5342676687179125, P43=0.5127961276310887, P44=0.5197475939379805, P45=0.4975207807890549, P46=0.5076331076226618, P47=0.5204678912248825, P48=0.5241038319462028, P49=0.5423467272052085, P50=0.5221445994270278, P51=0.48202418669231006, P52=6.732515426809341e-07, P53=1.5319085558643284e-06, P54=1.6867887103045312e-06, P55=3.5707135376979787e-06, P56=9.121027929253934e-06, P57=1.787208109464576e-05, P58=1.4154376879071552e-05, P59=0.00018791689028930579, P60=0.0018371969266959814, P61=0.0036815112399574018, P62=0.015897192436791975, P63=0.027496157874881348, P64=0.018579712024509098, P65=0.025825686203987615, P66=0.01951087490296569, P67=0.0069212781212693465, P68=0.0014484270669876602, P69=0.0013177192738311897, P70=0.0012274847323917478, P71=0.0008035524310750174, P72=0.0010787127574913699, P73=0.002138015387453084, P74=0.012314619065418501, P75=0.018336608723757922, P76=0.024261906430099345, P77=0.04059812364663292, P78=0.02373278350172946, P79=0.035844149047370086, P80=0.031708555890868954, P81=0.016915642433315028, P82=0.006830505387847617, P83=0.0022807878718785197, P84=0.0022323002065343703, P85=0.008192280933931767, P86=0.015204102057760897, P87=0.010419055468845586, P88=0.00526495612673791, P89=0.0065102220465804925, P90=0.01191905458308798, P91=0.037261201227488966, P92=0.03652773422185669, P93=0.012261928080927863, P94=0.007389364571599575, P95=0.010502781624036686, P96=0.00263138928292295, P97=0.00167933277140636, P98=0.0015746178649622448, P99=0.002354141722325652, P100=0.048657789454533806, P101=0.05525029897546431, P102=0.017057053429746914, P103=0.0010361676650363298, P104=8.373391306415296e-05, P105=0.00021880305263599094, P106=0.0035988116499908727, P107=0.002865818838092778, P108=0.0011855471211222799, P109=0.00010549185275556309, P110=3.841028142888193e-05, P111=0.00012213218986699206, P112=0.0018490660981584626, P113=0.0035723506166791246, P114=0.001715131339544299, P115=0.000394507786007263, P116=0.0003283627605683025, P117=0.003735324200353214, P118=0.0015551231000705077, P119=0.0013951476173302336, P120=0.00023529177971645445, P121=0.0006238913424260681, P122=0.0003928514701254498, P123=0.00021050847751553393, P124=0.00024366524486781188, P125=0.0004026181537940846, P126=0.0003581428226540941, P127=0.00017165216528262106, P128=0.0003419856526012492, P129=0.0008213436438491145, P130=0.0002733646029541587, P131=0.0005292833884529985, P132=0.0012754525320068787, P133=0.0005602090125330484, P134=0.00029415266136991245, P135=0.00011483921860113851, P136=0.00010686139281171182, P137=9.631956696751907e-05, P138=0.00012180627722492554, P139=0.00016376577492396778, P140=0.00013161022244648305, P141=0.00010480123777425845, P142=7.983263878037322e-05, P143=0.000131654119493943, P144=8.503412298497671e-05, P145=6.113789243711856e-05, P146=4.3350985551980944e-05, P147=5.590456009962284e-05, P148=8.682707143057902e-05, P149=8.364316083191056e-05, P150=0.00010758134391089672, P151=0.00018962927680602908, P152=0.0002049083441582003, P153=7.872116776789695e-05, P154=4.392890457680517e-05, P155=6.639515879154482e-05, P156=8.48422574925726e-05, P157=0.0001087093732994394, P158=8.727137677144694e-05, P159=0.00023814137995714228, P160=0.0006409304932829462, P161=0.00044100338366242283, P162=0.0011452541449215453, P163=0.001815290931457597, P164=0.0015524371380004822, P165=0.0006840511779784476, P166=0.000462237974803685, P167=0.00032635932688199464, P168=0.00028437900935688535, P169=0.00044492952647730646, P170=0.0003302404205763289, P171=0.00023570030445659886, P172=0.00018404752927626204, P173=0.00017222186618156039, P174=0.0002012606827215627, P175=0.00021409078928250164, P176=0.00016081893402940166, P177=0.0001208022821241655, P178=5.609464363693331e-05, P179=7.76313091104572e-06),Row( P0=-557.8564470012577, P1=29.188383175807946, P2=-21.68345906662947, P3=-6.936762711201776, P4=-19.580158093576554, P5=-13.985932247436672, P6=-4.262081488409907, P7=-5.355978629236806, P8=-8.135937137368542, P9=-10.206998508701002, P10=-9.140952211133103, P11=-0.7308285576842578, P12=-9.57473646745056, P13=-2.923966709614953, P14=0.9119724877178365, P15=-0.7809957008448476, P16=7.2758050756066295, P17=4.826308624789673, P18=1.267700416155532, P19=2.4980613343386975, P20=0.7364461415449177, P21=2.9153542093843043, P22=1.6069730636557236, P23=2.5340515088978854, P24=0.8038007941568899, P25=-4.293177830084233, P26=-2.9362314196489887, P27=-0.7873109454129508, P28=-1.0596499414883362, P29=2.955464604522866, P30=-2.1167872312193925, P31=0.7945893162225462, P32=0.5003329284148462, P33=1.1770100237273662, P34=-0.11765536339603522, P35=-0.25320479445170513, P36=-2.5870195063863637, P37=-1.1011427599341606, P38=-2.4437446277998087, P39=-0.13916679253847866, P40=0.6824948512923208, P41=0.6509863508157271, P42=0.6110221809373264, P43=0.5670722415380288, P44=0.5218339805525166, P45=0.5172002554833476, P46=0.5299411843931977, P47=0.5405309803728134, P48=0.559671028729499, P49=0.5587504104507388, P50=0.5363977199356089, P51=0.5990403215141156, P52=2.7834693918091864e-06, P53=1.0233814552533825e-05, P54=1.886920105820708e-05, P55=1.9082087114180637e-05, P56=1.8833796157692864e-05, P57=3.1420150190359614e-05, P58=4.171174413310493e-05, P59=0.00011914279809051419, P60=0.0005107857812677022, P61=0.0029920067445184206, P62=0.015319788031077837, P63=0.03102059058911347, P64=0.03514646577305537, P65=0.07359692339686755, P66=0.48105613609492964, P67=0.7353421137122, P68=0.18903691971995867, P69=0.11864719159608586, P70=0.08534485713194756, P71=0.01458047391826112, P72=0.003389618077491815, P73=0.006083797983368018, P74=0.012180992092256529, P75=0.01982200855207895, P76=0.048229882145924705, P77=0.0729846523428067, P78=0.03980830618001244, P79=0.02450301342497127, P80=0.020185259356429633, P81=0.03505866893481194, P82=0.11715080627865979, P83=0.1505520470667661, P84=0.20211100120658063, P85=0.2973841521707599, P86=0.1831132478233136, P87=0.3368327134259137, P88=0.6139992989002925, P89=0.2696422438167546, P90=0.05307637694642387, P91=0.016844307686365557, P92=0.016670030604735382, P93=0.01576192601294834, P94=0.02797901872213174, P95=0.06374390842641846, P96=0.058814095712580526, P97=0.16024651884613977, P98=0.24375383255229968, P99=0.14531007670143342, P100=0.14260338079526308, P101=0.03548131620921385, P102=0.005734130461145227, P103=0.010311290965353445, P104=0.03100075064354852, P105=0.02901304819676377, P106=0.02007548491251461, P107=0.008390733707781291, P108=0.007579352014581397, P109=0.008238702457558167, P110=0.00486191704128958, P111=0.006334685574150869, P112=0.005113421327970521, P113=0.004648963979572062, P114=0.010711759010781946, P115=0.02509913000462299, P116=0.08027105454576068, P117=0.039315049760050014, P118=0.010119969944751271, P119=0.0062955845705255, P120=0.008059459472165577, P121=0.006050984564781954, P122=0.010564535290298637, P123=0.008036151216076556, P124=0.003042394146203988, P125=0.006305611587580997, P126=0.010135257821305237, P127=0.0046489775024514445, P128=0.004526118034906717, P129=0.009238627049660286, P130=0.014163846679107335, P131=0.007756991214931302, P132=0.005224694144969788, P133=0.0028406079122871145, P134=0.002521886187050708, P135=0.0007500346265320892, P136=0.0009607022478981255, P137=0.0013649495327739125, P138=0.000936746391634252, P139=0.002284122542218223, P140=0.0036047206267341234, P141=0.002805745321113672, P142=0.0065887030478035385, P143=0.006634662453977596, P144=0.006158387989208133, P145=0.006210004656229217, P146=0.00433945842013813, P147=0.004526893705159564, P148=0.007012536809725039, P149=0.005421798211813191, P150=0.0031673533649919034, P151=0.0005848609569782506, P152=0.00011818779256144063, P153=7.184275490251537e-05, P154=5.925762359626102e-05, P155=8.509499464022308e-05, P156=0.00017758373873185968, P157=0.0003486070665911098, P158=0.0007631061261190329, P159=0.0009688421791541003, P160=0.0013172171529036267, P161=0.0011703330786324647, P162=0.0010385406741538978, P163=0.0009764365335314357, P164=0.0011692269396894725, P165=0.002388314175875571, P166=0.0030019008790871065, P167=0.002335064897492066, P168=0.003225104733049776, P169=0.0034833848626268913, P170=0.004257900550445971, P171=0.00397534091861965, P172=0.0026938511127006072, P173=0.003028327574963336, P174=0.0031068825853548553, P175=0.0035533966468305777, P176=0.002863500989743738, P177=0.002120377540726308, P178=0.0011415493610284968, P179=8.055967403607301e-05),Row( P0=-545.476960144662, P1=53.23798150134263, P2=-13.158455314000827, P3=11.271091915893477, P4=-4.282299447525114, P5=-7.7340528127581365, P6=2.590901953490801, P7=2.0377089284399776, P8=3.4571386448435604, P9=1.2611643397793248, P10=-2.6550435187027683, P11=0.7045872213287072, P12=-2.27684868264855, P13=2.391366615033093, P14=1.3711032649277457, P15=5.830406097063862, P16=4.225106380864584, P17=4.02855707329576, P18=5.264549198230564, P19=5.235563348320985, P20=4.054984578044159, P21=5.834478606814612, P22=5.938738955329231, P23=6.770607669674303, P24=3.887770443633748, P25=4.350836984969247, P26=1.9887229406240756, P27=3.4034013103604046, P28=3.646228254706071, P29=5.402523330186854, P30=2.3010764542851776, P31=2.109375412760657, P32=-0.5297969775506239, P33=0.2746324715117755, P34=1.0453151633851245, P35=2.1589003939446743, P36=0.7930208714269433, P37=0.863176567558785, P38=-1.1937693137857996, P39=1.833151056785931, P40=0.6817692764925191, P41=0.6774520074000007, P42=0.713386751393468, P43=0.6664347161957144, P44=0.6490579184144347, P45=0.6678135867715869, P46=0.7066208547771486, P47=0.6742985904803795, P48=0.6962961718460621, P49=0.7024142973176964, P50=0.7130959636591072, P51=0.7016318508981547, P52=1.0593415813043556, P53=0.13224439401898244, P54=0.01094041741802571, P55=0.0005508306289339394, P56=0.00033237220971534947, P57=0.000806167636824537, P58=0.0024511418358594468, P59=0.009196281571929723, P60=0.032914950086171305, P61=0.0595556990841309, P62=0.13190359278429295, P63=0.19290881355614226, P64=0.0741335482437827, P65=0.05715866233293713, P66=0.10072693895407885, P67=0.1067198156929269, P68=0.06677037307424473, P69=0.047836633463343554, P70=0.056390849353814235, P71=0.03392750252052678, P72=0.035973456922218525, P73=0.04044127635750548, P74=0.028178707286655093, P75=0.02804218626056638, P76=0.021394690277426662, P77=0.022405169339550078, P78=0.038139292241431616, P79=0.0573548135613011, P80=0.04719707020957713, P81=0.028266096645270113, P82=0.03362432825723537, P83=0.06206096106267114, P84=0.13271448406329833, P85=0.14908775815933878, P86=0.05554085005393415, P87=0.030913917743941605, P88=0.05229279403609551, P89=0.06333331124620115, P90=0.018295484667103026, P91=0.00473370187044359, P92=0.005584785575815589, P93=0.003241118953265721, P94=0.006671858624436725, P95=0.024078204324156773, P96=0.04499328359208331, P97=0.015036996821303678, P98=0.0029475286484780365, P99=0.0010510590267193555, P100=0.0005039813123232468, P101=0.0025324152185868484, P102=0.010139058918052724, P103=0.02608340500232772, P104=0.0042333665454666256, P105=0.0012364145694251072, P106=0.0020881220676379573, P107=0.0022015551736701694, P108=0.015117154104144714, P109=0.012255164433886573, P110=0.004597789642554287, P111=0.004950255604128709, P112=0.003194519068501332, P113=0.005249734240497119, P114=0.004864368680967026, P115=0.004316147708568407, P116=0.0043465896484474985, P117=0.009235356652071407, P118=0.002682714405058582, P119=0.0020876913276725896, P120=0.007508052476956867, P121=0.00436964578282483, P122=0.0015746866895420236, P123=0.002850328092583204, P124=0.005073993001845169, P125=0.003901396305392478, P126=0.0009732059517636218, P127=0.0034713041384538653, P128=0.0033876039018019754, P129=0.0013960281632447364, P130=0.0021714919586620918, P131=0.0021100131728820064, P132=0.0025218769142886913, P133=0.0021967711499959715, P134=0.004083114939136596, P135=0.005574652233573967, P136=0.0036187161658830487, P137=0.0033387513451993934, P138=0.0013383249550582804, P139=0.0012794284870017198, P140=0.0009638402555014712, P141=0.0008162672996075255, P142=0.0006402337746748366, P143=0.000507315307485654, P144=0.0004130081556231905, P145=0.00028448849999549, P146=0.0003834726203365895, P147=0.000298135330520135, P148=0.000344242862382493, P149=0.0005284296286660027, P150=0.000324313379348166, P151=0.00022227678922132616, P152=0.00015334040572522666, P153=0.0001325004159601071, P154=0.00013486464175254087, P155=0.0001205176646254331, P156=8.489034088053798e-05, P157=6.070336184547852e-05, P158=4.814755804900555e-05, P159=5.678130255934887e-05, P160=5.536579405067128e-05, P161=8.32615686247012e-05, P162=4.6911975356937596e-05, P163=5.462464615000065e-05, P164=5.264812689224842e-05, P165=6.32714821977411e-05, P166=6.141431349843472e-05, P167=5.731561449659879e-05, P168=4.0489451507371406e-05, P169=4.3838401693040324e-05, P170=3.650817286742845e-05, P171=3.93972623355227e-05, P172=3.4230373933203116e-05, P173=2.5760165892117504e-05, P174=2.085174060323347e-05, P175=2.272630422654333e-05, P176=2.0661554158934914e-05, P177=1.9883159454588855e-05, P178=1.0844599081131636e-05, P179=1.4173036622769306e-06),Row( P0=-595.4246764017494, P1=57.690328370635875, P2=5.167144880690533, P3=9.808011526008576, P4=2.781303296059555, P5=-3.048114064128029, P6=2.9459208335221394, P7=6.835554668245023, P8=5.201852440884232, P9=-1.7939175413721042, P10=-0.6073019911313546, P11=2.067175466953693, P12=-3.4313509631643146, P13=-0.05547934767288529, P14=1.121938950416138, P15=0.5046040868917956, P16=0.6305123061759168, P17=1.6496189197617812, P18=2.302900562704705, P19=1.820508714941639, P20=0.9536865127490451, P21=2.7910445599317777, P22=2.6129511303983444, P23=4.05686398495548, P24=6.278253773832555, P25=11.255050278239333, P26=7.830728700443247, P27=10.944791157525684, P28=6.6633405174412905, P29=4.483624658952116, P30=1.4060725815772486, P31=1.8787992395366973, P32=-0.34929875983379155, P33=2.886567292712142, P34=-1.6084685474510894, P35=2.5118572411291336, P36=-1.9132906088202613, P37=-1.431627412959455, P38=-1.7293738379426244, P39=-0.36107100542539183, P40=0.70110642047435, P41=0.6703543003095583, P42=0.7038565967444828, P43=0.8225421337853892, P44=0.8523516627587289, P45=0.8104431180421471, P46=0.7313708225480039, P47=0.6670040180183782, P48=0.6355849717836238, P49=0.6647374753591527, P50=0.7225934583921481, P51=0.7486919926239668, P52=0.22797506744644727, P53=0.0473449085689821, P54=0.0038194116235466587, P55=0.0005925867814212124, P56=0.0005820428993284139, P57=0.0006112656046881759, P58=0.0006961324599104575, P59=0.007822441402588774, P60=0.11047032575231025, P61=0.3670452631583788, P62=0.3490798295406675, P63=0.08573048710460841, P64=0.01220731817318562, P65=0.020425362639927946, P66=0.027228308113465525, P67=0.01638076433001158, P68=0.008671919948299519, P69=0.016162904875857753, P70=0.06120468489694915, P71=0.07764464811759761, P72=0.06613954397683271, P73=0.05385627066853421, P74=0.023064340391124438, P75=0.006682155287808309, P76=0.003878915851657218, P77=0.00447916314070035, P78=0.005153851702824269, P79=0.015497136346446156, P80=0.06256656477710155, P81=0.06676152551704012, P82=0.05751353777237293, P83=0.038422824015064076, P84=0.050320996867127925, P85=0.041240845126098505, P86=0.0089920703376672, P87=0.005967410214015352, P88=0.025088889530570896, P89=0.034628295657041175, P90=0.022415543114438677, P91=0.013937420644522644, P92=0.014140290487040602, P93=0.003740452760197078, P94=0.009369388550006673, P95=0.0209587071071954, P96=0.010497078745403808, P97=0.002748483994111673, P98=0.0015179649515910307, P99=0.0007869907877007592, P100=0.00218073690333517, P101=0.005120274622174912, P102=0.002700223469464468, P103=0.0008018343759440261, P104=0.0004150075173788301, P105=0.0014535616803923019, P106=0.0023927350518145817, P107=0.0014041745383262453, P108=0.0010192877307317117, P109=0.0012759771452443697, P110=0.002510499285124589, P111=0.003701793487433193, P112=0.0016151571831967838, P113=0.0017944205059475355, P114=0.0017745236472529356, P115=0.0009845705629601125, P116=0.002021176311281594, P117=0.0008249722732858767, P118=0.000557359461427449, P119=0.001618840098149709, P120=0.0011266189154990806, P121=0.000886652200706294, P122=0.001139580077033026, P123=0.0005984418753300953, P124=0.0013619470682491653, P125=0.0008944490851986064, P126=0.000496214226089795, P127=0.00103601225071848, P128=0.0006449342870049415, P129=0.0005220555950700011, P130=0.0003066949870550358, P131=0.00033389574494725326, P132=0.0003442743236712761, P133=0.00033192089723808876, P134=0.00033940690355251346, P135=0.0005280049607074845, P136=0.0005558218503342247, P137=0.0004159556626782777, P138=0.00017290817945513778, P139=0.00020635456319026847, P140=0.00015630045902844295, P141=0.0001436540270652621, P142=0.00014245863613129042, P143=0.00017421762566406857, P144=0.0001999492875449705, P145=0.000254677809702345, P146=0.00025444788748296097, P147=0.00020689062677044028, P148=0.00021321540434791002, P149=0.0001316533434123166, P150=0.00015386184697776194, P151=0.00027566792609379405, P152=0.0004318845222507337, P153=0.00047095026075215737, P154=0.0005375967903558491, P155=0.0006115978865674055, P156=0.00048207902685737717, P157=0.00037735555648374787, P158=0.0003949488482260538, P159=0.00026065752302495106, P160=0.00020482963280419724, P161=0.00017171803847409702, P162=0.00021543916451409806, P163=0.00016150631339252063, P164=0.000139917062804289, P165=8.558945392470141e-05, P166=9.682679632587415e-05, P167=0.00012865321988767377, P168=0.00016429865121032983, P169=0.0002022546207167089, P170=0.00014058936931352967, P171=9.965447742854112e-05, P172=6.217882445017958e-05, P173=0.00011884229195350323, P174=0.00016843649594621286, P175=0.0002083852702218795, P176=0.00028339931239275456, P177=0.0003504930366090816, P178=0.00020879940700641912, P179=2.637447525712021e-05),Row( P0=-625.1968072410325, P1=70.80577696124331, P2=-13.050909665605007, P3=15.749517183702247, P4=-1.0402976915263185, P5=-8.82401377748419, P6=2.8106176818923387, P7=2.945615949154361, P8=1.7523982676011305, P9=0.6899144882998799, P10=-3.120574741974352, P11=-2.257417801838837, P12=-1.3870522906570435, P13=-0.9985793817406186, P14=-1.9353328591317265, P15=1.8745469968815245, P16=2.4616713681930773, P17=3.4751161253873666, P18=4.274957510506047, P19=2.4079070546465333, P20=1.1046033595793363, P21=-1.6859862244599002, P22=0.7893673954967951, P23=-2.001993942950468, P24=-0.42743161944907193, P25=1.599740818642336, P26=1.2280815501035462, P27=3.024456504210475, P28=0.6848127837920122, P29=3.8859362592377362, P30=-0.3615046594887494, P31=2.760129781867966, P32=-0.3511311932773682, P33=2.712282507520434, P34=1.5232804020252797, P35=6.229539842566246, P36=4.195561160855596, P37=4.78910066042166, P38=2.7319473362222073, P39=2.452013462116049, P40=0.7222066090732363, P41=0.6415341961414625, P42=0.6600018512882173, P43=0.6640421950451265, P44=0.6506782934641406, P45=0.6864629525362587, P46=0.6503593802932985, P47=0.5990012797331539, P48=0.6117927941207916, P49=0.65936349469703, P50=0.7481370681379562, P51=0.7916186366017139, P52=0.0006019024816234416, P53=0.00021550020575254167, P54=9.091713471332579e-05, P55=3.43071931845447e-05, P56=7.679227521764843e-05, P57=0.0018089061646414874, P58=0.009650397630149788, P59=0.013433004375280645, P60=0.02675521852319901, P61=0.061917021015845015, P62=0.050324891815124215, P63=0.010145604204583061, P64=0.003337487356732625, P65=0.018802941481731457, P66=0.02903551145169878, P67=0.02258790614148858, P68=0.031004841717283443, P69=0.03281876169469655, P70=0.02669867455443871, P71=0.01977831309599515, P72=0.02797251056108529, P73=0.02169235952819681, P74=0.00711901623351183, P75=0.00484726589398532, P76=0.004545940581969895, P77=0.010462496422854425, P78=0.013759833542834377, P79=0.023908054286499336, P80=0.03974628351276707, P81=0.051629583727261924, P82=0.09374757480356055, P83=0.10320312908323473, P84=0.05372492496389894, P85=0.014547284178341006, P86=0.017314485211447257, P87=0.0248487907742734, P88=0.046213862840774354, P89=0.022317046280562285, P90=0.006446221079799503, P91=0.01665986625525999, P92=0.02267879147332161, P93=0.02957860937490751, P94=0.034930088826640386, P95=0.046274432983958697, P96=0.011942934107684386, P97=0.007009970567293812, P98=0.020396914935584202, P99=0.003897267730103048, P100=0.0018903772600622035, P101=0.0028022309989468694, P102=0.0024358677244625043, P103=0.0011571728116720132, P104=0.0031765464699439494, P105=0.003457529507687966, P106=0.004214462531525515, P107=0.0014248435847202911, P108=0.0037545068092799177, P109=0.008469068544452736, P110=0.020229251786616467, P111=0.01395718765881656, P112=0.0047029025244434905, P113=0.004250715642871545, P114=0.004371156409359154, P115=0.006917884844866459, P116=0.0024152236922575453, P117=0.005613777136607531, P118=0.001622077106960496, P119=0.00540791825394709, P120=0.004035212687958614, P121=0.001862874926566449, P122=0.0019274559702206068, P123=0.002678131759403445, P124=0.00253084102950163, P125=0.0017704065848159502, P126=0.0016075511874607243, P127=0.0027529985276106136, P128=0.0018733621722234863, P129=0.001309819405715315, P130=0.0020059209058663754, P131=0.002762804316650178, P132=0.00344839425491456, P133=0.003663563695517302, P134=0.0037287705163696876, P135=0.0018152768079468086, P136=0.001688115734905458, P137=0.0008057479399861288, P138=0.000868733532023658, P139=0.0005120983068890519, P140=0.0005994096388668626, P141=0.0008533560473429606, P142=0.0004591024123182117, P143=0.0003148589103016943, P144=0.00022783795698885485, P145=0.00017254763112682376, P146=0.00011457714390274734, P147=0.0001827161652174508, P148=0.00024021509283068813, P149=5.985439304884079e-05, P150=5.4025817582623766e-05, P151=7.131814577264922e-05, P152=7.555114352556346e-05, P153=0.00012318145534705632, P154=8.906744603226924e-05, P155=8.937101944312065e-05, P156=0.00014464892424045014, P157=0.00029494610808157015, P158=0.0003699592908299952, P159=0.0001319240256787971, P160=7.777619708529509e-05, P161=7.245361799541563e-05, P162=0.00019419862644147028, P163=8.278355660295315e-05, P164=3.769438220846422e-05, P165=8.497528938949149e-05, P166=8.28560030258782e-05, P167=7.945169435025655e-05, P168=6.35114478736205e-05, P169=8.0543680818823e-05, P170=4.226235598976468e-05, P171=2.5006463997711995e-05, P172=1.6918443476772225e-05, P173=1.2104770840406996e-05, P174=1.3265752580577036e-05, P175=1.720502917047384e-05, P176=2.7441199510287015e-05, P177=2.637481463160362e-05, P178=1.4875286913083248e-05, P179=2.6935137808537712e-06),Row( P0=-580.8136150500102, P1=84.07182191405764, P2=-11.97509626679123, P3=12.762823332300204, P4=0.32235160437460075, P5=-5.684460649408057, P6=6.36069582853123, P7=7.772559399779679, P8=9.855217120284058, P9=4.090990025988374, P10=0.6806247486095838, P11=7.817890546310357, P12=6.675943052099172, P13=5.9193462816485445, P14=4.58280944037006, P15=6.038197534645596, P16=5.499308831209338, P17=5.971463899361937, P18=4.635521962562985, P19=6.962828780894708, P20=5.045822774889307, P21=1.6197089338112376, P22=2.7385565651932415, P23=-0.36864574869875355, P24=-2.35294404494865, P25=3.7793753100346397, P26=2.0901520013964943, P27=5.390360901273596, P28=2.92440154225871, P29=6.44897994264186, P30=1.5996112068270927, P31=1.083091353605267, P32=-1.1883950307209954, P33=1.1196713123430004, P34=-1.7135270985500661, P35=4.061885817422633, P36=4.293693562129452, P37=5.843035322603681, P38=3.905874501963319, P39=2.99481527932819, P40=0.6065370211310771, P41=0.6600495111866841, P42=0.6691730576423527, P43=0.6796074646102324, P44=0.7052191508632564, P45=0.6932978602534937, P46=0.7143631765919763, P47=0.6564895254209453, P48=0.6799484823031313, P49=0.7558601848559171, P50=0.6852260428582778, P51=0.6158834590118067, P52=0.050954668044598146, P53=0.005397211188225929, P54=0.001843563732701615, P55=0.00019884634094004982, P56=0.00032635646037751186, P57=0.0045816708343371735, P58=0.013612143997576747, P59=0.013699414710567412, P60=0.03572957644915951, P61=0.04935454147236921, P62=0.03128201555364295, P63=0.024694720816754688, P64=0.015906600180257052, P65=0.040010771573443196, P66=0.02444607943943871, P67=0.01464091962200121, P68=0.02863348961257387, P69=0.025963323510448822, P70=0.019532503932478913, P71=0.010360127083011391, P72=0.011261723833031761, P73=0.010843572472783811, P74=0.016273065686016542, P75=0.019996768783298924, P76=0.009470610929412935, P77=0.0152544161055827, P78=0.03905666568311047, P79=0.10153048035870708, P80=0.08034172011290573, P81=0.025685086436813637, P82=0.016979565228043465, P83=0.008964472285276361, P84=0.008991019800704696, P85=0.04500672348307126, P86=0.10167759725647774, P87=0.05404297863095227, P88=0.02774983084093088, P89=0.014140892626405971, P90=0.012112104018980385, P91=0.06787994449949614, P92=0.09820179291867706, P93=0.022023995339653712, P94=0.04466277811629059, P95=0.07989240297843188, P96=0.039966678703365795, P97=0.023520525755569573, P98=0.00518380741566616, P99=0.0024946578347809405, P100=0.012462552903592123, P101=0.012826403973694899, P102=0.0037446470892529616, P103=0.00176850702304483, P104=0.006677854086927425, P105=0.006034671142303219, P106=0.006869936164467691, P107=0.00558502428150027, P108=0.0047841074322942665, P109=0.012080483308789037, P110=0.02042209110210115, P111=0.011443371379218841, P112=0.006133380284407394, P113=0.009263059219725384, P114=0.005647079122160843, P115=0.006637722579756609, P116=0.00731265118330745, P117=0.00474035903535798, P118=0.002339346191129202, P119=0.0033706331135507435, P120=0.006808309743654721, P121=0.0035094696265934875, P122=0.0033145210037882084, P123=0.003085324259343474, P124=0.0023343814647552186, P125=0.0018471111400123303, P126=0.0008452423707160585, P127=0.0008898954602297762, P128=0.0014720984401773523, P129=0.0018777258729733414, P130=0.002408440327279629, P131=0.002228210740615152, P132=0.00174495713938488, P133=0.0011944890878805192, P134=0.0012077639912030222, P135=0.0008759303031395972, P136=0.0003772018068854053, P137=0.0004586225139321613, P138=0.0003031057581375812, P139=0.0003210724359824706, P140=0.0002846940900101409, P141=0.00027312764396563143, P142=0.0003351643688047586, P143=0.00020227893018665995, P144=0.00018071203604107792, P145=0.00012170572032912906, P146=0.00015045949524781466, P147=0.00012376686004440842, P148=0.0001402039320759991, P149=8.696353739437522e-05, P150=3.650593943040457e-05, P151=5.285690311562063e-05, P152=5.390540018351753e-05, P153=9.927781132772559e-05, P154=7.507620175396186e-05, P155=4.8146126191631495e-05, P156=4.758455627868278e-05, P157=8.240396383282223e-05, P158=8.056439202333663e-05, P159=6.346922307970571e-05, P160=6.905713397593266e-05, P161=5.182434310450002e-05, P162=4.66946294296302e-05, P163=3.5300458015565244e-05, P164=2.7887064826707627e-05, P165=2.5476845670965856e-05, P166=2.1857402609081918e-05, P167=2.043128097103386e-05, P168=1.6893714201795227e-05, P169=1.8319558427225104e-05, P170=2.6724366831571758e-05, P171=2.751069007441007e-05, P172=1.8694308510725604e-05, P173=2.4134345985723656e-05, P174=2.9738432335433176e-05, P175=3.1330177919417846e-05, P176=2.0851042187688843e-05, P177=1.855212579065799e-05, P178=1.3893183433667894e-05, P179=2.937842891687251e-06),Row( P0=-540.9158872083801, P1=52.022854895021936, P2=-4.31856420648195, P3=18.700210843019914, P4=1.8285330082197313, P5=1.040115426179531, P6=9.716497290814319, P7=5.674183060938006, P8=5.91440375444379, P9=9.135637982117117, P10=1.503238599140696, P11=7.289967408042819, P12=1.5489749048994181, P13=3.1687121290467575, P14=1.6844237647545484, P15=3.127666908408384, P16=5.179738956369239, P17=7.78376785747737, P18=10.523939658522293, P19=13.426609062117175, P20=10.84388636079753, P21=6.2736504423428325, P22=4.8082573301463025, P23=0.5667584475616392, P24=-1.5098722861108465, P25=5.352643711756171, P26=1.2837242626529084, P27=3.572585105400646, P28=1.5342061690070283, P29=0.2510305643141632, P30=-0.9286040725621802, P31=2.049724885289447, P32=1.0825611803883928, P33=3.2300627634238386, P34=-0.9126617185750192, P35=1.8865717231782984, P36=-0.6052084747911008, P37=2.7712495917836777, P38=1.2849347888341986, P39=0.9203671679426758, P40=0.6367588771084359, P41=0.6356074087652599, P42=0.6733733948660001, P43=0.7343126529131621, P44=0.7088983775789116, P45=0.6799236325660926, P46=0.6726766238824449, P47=0.6560284588495346, P48=0.7275653332903723, P49=0.7317400593088254, P50=0.7167897405793623, P51=0.6675136597764781, P52=1.5009128669319793, P53=0.1955729942895846, P54=0.015564886192142163, P55=0.0007959836953033077, P56=0.00042409150254815476, P57=0.0008622125089062742, P58=0.0029429614613328244, P59=0.005048459641288715, P60=0.009263450629631327, P61=0.016786613348410813, P62=0.05290539908902551, P63=0.11259591086574233, P64=0.1486943286039054, P65=0.42599483724698695, P66=0.3806912252409763, P67=0.12775542053822556, P68=0.03322293302103374, P69=0.009297008369979235, P70=0.0066907806135024245, P71=0.0030452951681377315, P72=0.002739054685305812, P73=0.0036321271971257517, P74=0.009127763231093941, P75=0.02243451044557598, P76=0.09652476669624588, P77=0.27226394137116183, P78=0.4147860628847258, P79=0.46980797461478985, P80=0.26546737310151364, P81=0.1757092641277443, P82=0.14437521275747803, P83=0.1796388023076622, P84=0.2141687143204652, P85=0.05279986939661758, P86=0.011612214443153851, P87=0.018284300928204492, P88=0.05735165511832632, P89=0.08451495589432592, P90=0.21679141408368333, P91=0.4303273490234546, P92=0.253962009853562, P93=0.11726872786538825, P94=0.04800932428819556, P95=0.05122310514597644, P96=0.03129543977126323, P97=0.011696167360252419, P98=0.024689723103415544, P99=0.12121918586597397, P100=0.169376690300218, P101=0.15231622401546976, P102=0.11688082589886252, P103=0.028126430060871434, P104=0.02076235284339177, P105=0.07651920833222296, P106=0.07334314331240317, P107=0.03495549760774378, P108=0.1240389640030944, P109=0.0611896724955436, P110=0.016275908246717784, P111=0.024325434286576127, P112=0.05150114725868605, P113=0.046478366591829606, P114=0.03581250404640306, P115=0.007707859347857827, P116=0.012959944225755464, P117=0.0349966333101154, P118=0.004948271755176056, P119=0.011205186831310837, P120=0.015613310876065071, P121=0.006877031929786817, P122=0.002625088326878806, P123=0.006175456507170125, P124=0.027257785049232836, P125=0.008824719619164277, P126=0.0017766706861582972, P127=0.003938809913702689, P128=0.0020438031521736177, P129=0.0024102092317290187, P130=0.006598016608212715, P131=0.004226653341822012, P132=0.006472402056528281, P133=0.015473186339618611, P134=0.004713552164918579, P135=0.01049907328921076, P136=0.005595316303455989, P137=0.0016008498979651396, P138=0.001999128265304033, P139=0.0009947523278732032, P140=0.001969518087385436, P141=0.001991780590630073, P142=0.005743833825358077, P143=0.005361004813925719, P144=0.008287786436501505, P145=0.0029676591277944956, P146=0.001576368605238743, P147=0.001374765703776975, P148=0.0005767097952421202, P149=0.00032128605155460255, P150=8.735218063550951e-05, P151=5.7520054874835604e-05, P152=6.66406612540332e-05, P153=0.00021508501005313108, P154=0.00023187645562393647, P155=0.00015544105999745034, P156=9.682135124252047e-05, P157=8.631310408268949e-05, P158=5.194396172443722e-05, P159=6.076969415567033e-05, P160=9.362761309935228e-05, P161=6.84740658340498e-05, P162=7.482714547507878e-05, P163=8.551403133246204e-05, P164=0.00011931155076908398, P165=0.00012521925005121428, P166=9.694692955865954e-05, P167=7.92499390010317e-05, P168=7.461531475321443e-05, P169=5.435303018684318e-05, P170=4.3674525413914696e-05, P171=3.6592314835761904e-05, P172=2.522730623010573e-05, P173=2.0986557135137916e-05, P174=2.3247017766596858e-05, P175=3.6372161424857764e-05, P176=4.211926261024658e-05, P177=3.1913714620866406e-05, P178=1.6879066636207263e-05, P179=2.4098138317206896e-06),Row( P0=-648.8573253459535, P1=77.10867235560814, P2=1.4524193955400138, P3=13.233909767734163, P4=-2.478192279405848, P5=-0.8314825732644107, P6=-0.5022967535602886, P7=2.418582661129701, P8=2.2907545230913477, P9=-0.8541184116687316, P10=0.894016677700298, P11=0.5619343602067685, P12=-0.7015559103408945, P13=2.6624057633066944, P14=-5.404710292994685, P15=2.3965270250253674, P16=-0.6898866912690834, P17=0.6662238550524566, P18=2.4596065818574298, P19=1.608179499343484, P20=-0.1442282676234133, P21=-0.40112039182097137, P22=3.196992079653893, P23=0.46972751176536104, P24=3.494269813863189, P25=5.129286809268027, P26=3.4381901600547566, P27=9.248604519511268, P28=4.538584605530515, P29=8.48073944756578, P30=4.176289089734268, P31=6.7569663867187915, P32=1.8039425866291212, P33=5.452374032804232, P34=1.452968992509731, P35=2.3292229015894574, P36=0.9154789266642714, P37=1.227926939164223, P38=0.2776136517722705, P39=0.6810981752807387, P40=0.6447595469497417, P41=0.6798803603295321, P42=0.6960712264237013, P43=0.7065298902631616, P44=0.70929696281081, P45=0.6874290270333159, P46=0.6488309879705718, P47=0.6164445467775288, P48=0.6524017037687583, P49=0.6635443192595016, P50=0.6447481042642138, P51=0.6466403310163105, P52=0.013276628860057267, P53=0.0006860094847867331, P54=5.488047531359358e-05, P55=1.9068103995222296e-05, P56=3.381486015851135e-05, P57=7.374304205057215e-05, P58=0.0008619275368808335, P59=0.010823876501336005, P60=0.0390497464169648, P61=0.06544191565595482, P62=0.0577913224893679, P63=0.010917943294519325, P64=0.0006850040918268136, P65=0.0011683306511636733, P66=0.0028180081072375317, P67=0.005093701094670069, P68=0.010378555857568524, P69=0.015483076684611765, P70=0.013043758579352126, P71=0.017330372440240667, P72=0.025648864361399785, P73=0.021496341735563834, P74=0.004969987727657677, P75=0.000795928621105959, P76=0.0013041665948167731, P77=0.004404845456186204, P78=0.008735359388459494, P79=0.008486569240826377, P80=0.0040049924951609354, P81=0.0030143797916026764, P82=0.0036718281559980984, P83=0.005343011984932519, P84=0.00929331745764068, P85=0.007106615094001799, P86=0.00368809269373206, P87=0.0035944014174293013, P88=0.0027222020877092842, P89=0.0015383090861627057, P90=0.00195490232822386, P91=0.003764056918776296, P92=0.004242168596625033, P93=0.0022084911289101954, P94=0.0015894530994964107, P95=0.002061095707595988, P96=0.001937661304263625, P97=0.0032228359107597383, P98=0.004699424086398909, P99=0.0013131432664782677, P100=0.000640369394244632, P101=0.0004576128155729266, P102=0.0006222457253797315, P103=0.001005278500334489, P104=0.0009411451861812962, P105=0.0005210959749938358, P106=0.0006130586287555009, P107=0.000577964024372731, P108=0.0013572316950592274, P109=0.0007856322493952477, P110=0.0007188874913214197, P111=0.0008748784277310559, P112=0.0005660083675117705, P113=0.0005328747164058157, P114=0.00053157847908815, P115=0.00039195120249409753, P116=0.0005857199748378015, P117=0.0005596712369795703, P118=0.00014725970064341336, P119=0.0003155517111077603, P120=0.00038208953552209787, P121=0.0001732669359294966, P122=0.00011543190823203808, P123=0.00014409800166928548, P124=0.0001322372646651487, P125=0.00013034349334260965, P126=0.00010984175109538529, P127=0.00022824698296409533, P128=7.8730165432197e-05, P129=5.329697536448406e-05, P130=6.894637287475058e-05, P131=6.744800358679279e-05, P132=8.67061760927261e-05, P133=0.0001072838925841288, P134=0.00015186603337134162, P135=0.0001691107344932658, P136=0.00028317460057469395, P137=8.824526125316683e-05, P138=3.893828931671034e-05, P139=2.493900407052502e-05, P140=1.713050865648414e-05, P141=1.7641863635061228e-05, P142=1.610200913959245e-05, P143=3.4991507644254586e-05, P144=4.6029878104219325e-05, P145=6.211237939089182e-05, P146=0.00010680485928486783, P147=8.745286827638473e-05, P148=9.424630146729812e-05, P149=5.4931569763371804e-05, P150=7.310244974009581e-05, P151=8.828682629252991e-05, P152=0.00012418991809652128, P153=0.00014997438744607843, P154=0.00014419602756786135, P155=0.00014229585745510515, P156=0.0001058861341067106, P157=0.00010348745430808344, P158=8.92688245093471e-05, P159=5.6720946453396815e-05, P160=4.060615057813877e-05, P161=4.7267012531314684e-05, P162=7.781519354553083e-05, P163=8.259424088937175e-05, P164=4.366619448552482e-05, P165=4.596737177138421e-05, P166=5.7947751480606054e-05, P167=5.027749906792096e-05, P168=3.344489673237742e-05, P169=2.1650723478043744e-05, P170=2.0210495559909326e-05, P171=1.321733245439714e-05, P172=8.567947361182914e-06, P173=1.150166220841411e-05, P174=1.6070506410275632e-05, P175=1.813227863342374e-05, P176=2.140674516299669e-05, P177=2.3536462777984245e-05, P178=1.2024442066363457e-05, P179=1.3579614047850123e-06),Row( P0=-671.0341783498395, P1=84.60649726143352, P2=-3.174965897162384, P3=10.592838616451084, P4=0.22801386741699178, P5=-1.6081331778279093, P6=5.111499272165366, P7=2.704858188282945, P8=2.061156235313642, P9=4.5276342953255115, P10=2.203577191300605, P11=0.7092990406002809, P12=1.3318111297322663, P13=1.2182920731989484, P14=-3.151351344259925, P15=2.679339311262707, P16=1.0007068887631285, P17=2.4490681086217054, P18=1.7761007360994114, P19=1.5528242685678137, P20=0.06739635993444754, P21=-1.7352264598198166, P22=2.5612419677782094, P23=0.9354275532423986, P24=-0.006286254115511749, P25=2.0201046649496877, P26=-0.20603804245660068, P27=3.6438678230256656, P28=0.24359639011805267, P29=4.444510659489139, P30=1.5884107813345816, P31=2.7418837305545036, P32=0.4497548933706986, P33=3.3057378488801104, P34=2.5239525949917603, P35=5.3007551478176005, P36=3.127502121999016, P37=3.677125937119044, P38=2.297051720494259, P39=2.1280870707353623, P40=0.7335957043400582, P41=0.6878563210426634, P42=0.6937068948348103, P43=0.7090930898005021, P44=0.690110684958355, P45=0.7004746102133134, P46=0.7209227171699818, P47=0.6748832122443225, P48=0.696105752167724, P49=0.7301145858080309, P50=0.7529290834590794, P51=0.7538599697106906, P52=0.000623331914927963, P53=0.00016448128533720714, P54=8.424751296532043e-05, P55=3.52863298508924e-05, P56=6.036973060592863e-05, P57=0.0009760856053518647, P58=0.006361063488568445, P59=0.01109900559947095, P60=0.01594970515210919, P61=0.020451738667066725, P62=0.021540779755837867, P63=0.00720015509159134, P64=0.0011913020599234447, P65=0.005057931487281305, P66=0.010778988279025573, P67=0.008629981216707083, P68=0.006762853395019921, P69=0.007881839658258121, P70=0.006599180634274512, P71=0.0033003370163514492, P72=0.008172201444014876, P73=0.012135670387470415, P74=0.006617053759711617, P75=0.0028427170555256017, P76=0.003085535399755965, P77=0.005146327768654095, P78=0.006138961333428128, P79=0.006753124388786881, P80=0.004066226196544406, P81=0.006095924569628054, P82=0.01113406929854783, P83=0.019092848514936582, P84=0.027481337948120656, P85=0.015381728347565961, P86=0.002451278941241154, P87=0.005846925692837948, P88=0.006427231524290714, P89=0.0015363451641564525, P90=0.00040621552676342723, P91=0.0005924280806836813, P92=0.0018574717093147084, P93=0.0036745642275535107, P94=0.03338339609478662, P95=0.015130474739834992, P96=0.0012789325727501862, P97=0.0011180961249567518, P98=0.000690111808815088, P99=0.003982030210621488, P100=0.01056656305310557, P101=0.0016427570257233931, P102=0.00022467998945294995, P103=0.00032219636401290924, P104=0.002890649820408956, P105=0.004274595237608848, P106=0.0013212830927025756, P107=0.00017631639199082252, P108=0.009004831727852583, P109=0.011084341248128697, P110=0.001067112344649286, P111=0.00027979487434360927, P112=0.0017042272750109578, P113=0.000987682307090053, P114=0.0002118035021798812, P115=0.000471504954187003, P116=0.0006025106362748018, P117=0.00033725554914152875, P118=0.00017564956049615833, P119=0.00023803084784621858, P120=0.00036776871744074384, P121=0.0002486630531314789, P122=8.180210921311589e-05, P123=0.00015707287432910173, P124=0.00021276346004022148, P125=4.515144163319604e-05, P126=6.121457084204205e-05, P127=7.413378424123138e-05, P128=5.717878513507186e-05, P129=7.292496647641562e-05, P130=9.7717909839304e-05, P131=0.00013080956034835933, P132=9.581377397954106e-05, P133=0.00015781230277801326, P134=0.00014799915905916737, P135=0.0002682932848611875, P136=0.00013299983280163776, P137=9.157834452043788e-05, P138=6.113952225104764e-05, P139=5.200307279379917e-05, P140=4.2066581378837094e-05, P141=4.381554295728746e-05, P142=3.462422633679757e-05, P143=4.212531820865579e-05, P144=5.233192285199908e-05, P145=3.6872744040520644e-05, P146=1.809053880553565e-05, P147=2.2357876075064337e-05, P148=2.3706412125830038e-05, P149=1.164955648589857e-05, P150=4.068028446447693e-06, P151=8.394737299410703e-06, P152=1.851067449802202e-05, P153=1.6041487273379684e-05, P154=1.782070826992553e-05, P155=5.494396186360088e-05, P156=4.041148581307542e-05, P157=2.8816584537973743e-05, P158=3.28755601566791e-05, P159=1.4110100172398238e-05, P160=1.0002707979038598e-05, P161=1.4687223382634998e-05, P162=8.192394691110919e-06, P163=5.2128935513135925e-06, P164=5.1333130165556395e-06, P165=7.373747527698646e-06, P166=4.8018803955557585e-06, P167=3.786605354239776e-06, P168=3.701458968059622e-06, P169=2.6440338694849733e-06, P170=2.935597339329094e-06, P171=1.7525650824204472e-06, P172=1.4761495525136398e-06, P173=1.5251040479842563e-06, P174=2.6683972429978223e-06, P175=2.124834182896047e-06, P176=2.1935733256340437e-06, P177=2.3989526085109543e-06, P178=1.2796195802781888e-06, P179=1.478144734338971e-07),Row( P0=-513.5409521436064, P1=52.82386418020799, P2=-16.465913110869945, P3=11.685038631799271, P4=-5.440900964814009, P5=-10.839249616521435, P6=3.9214617704921886, P7=-0.09530428196024421, P8=0.24665417806968423, P9=2.3356598509703232, P10=-6.302561895051878, P11=0.705006031711525, P12=-2.0685765868335984, P13=-0.9918114250207347, P14=-2.222028193143967, P15=-0.5481184269542358, P16=0.12082596329279066, P17=-0.7074287494648364, P18=3.968761929724098, P19=5.506673963420121, P20=5.850717238403823, P21=2.8103110039414085, P22=4.56226235917758, P23=2.641179309367471, P24=-0.4283125512339088, P25=2.6124311721505182, P26=-1.7985627300717721, P27=2.061632691854686, P28=1.701405776919755, P29=2.7558327761560983, P30=0.47231203050903153, P31=2.5516155516786543, P32=0.07260030439254697, P33=2.054238092991678, P34=-1.2512053370783265, P35=2.650162992657599, P36=1.3508960064074795, P37=3.152783657201212, P38=2.114050328988354, P39=2.231062680016825, P40=0.6560869150166587, P41=0.6626453935595747, P42=0.6297080648226387, P43=0.6256699236858231, P44=0.6423253781639277, P45=0.6691950392266653, P46=0.7098628070152823, P47=0.6826583486481734, P48=0.7044095217761881, P49=0.716246053642913, P50=0.6884061227575344, P51=0.6659968647103847, P52=1.255392355321245, P53=0.19575515548955907, P54=0.010343455893618953, P55=0.0004617352039692723, P56=0.0005792350667606554, P57=0.0032576324657641238, P58=0.009734131567564105, P59=0.01769648811092808, P60=0.02862013681492963, P61=0.04293624371286645, P62=0.1137107730335235, P63=0.17288189903574067, P64=0.12433664724648513, P65=0.07153182648861552, P66=0.03697627880565702, P67=0.02729856928680378, P68=0.01974982339418242, P69=0.022782800654865952, P70=0.021733109488733377, P71=0.012305640193003717, P72=0.017640661658791336, P73=0.03071348694369889, P74=0.05089801952946163, P75=0.05933148093020599, P76=0.09731198980061041, P77=0.46373352172580073, P78=0.6715823097925308, P79=0.3368590038715633, P80=0.12610010895329862, P81=0.06557122239865995, P82=0.030625483011065595, P83=0.035348058074112426, P84=0.07216707579731518, P85=0.10325046691037694, P86=0.06835780911721172, P87=0.06612506207207984, P88=0.06054486686425379, P89=0.06899287556991467, P90=0.11119903014235795, P91=0.11413784242165713, P92=0.1772148262738925, P93=0.1540919724304993, P94=0.2861453541731494, P95=0.19636317237807263, P96=0.09047514701457364, P97=0.07286376406032669, P98=0.0448552210317933, P99=0.019129206046024035, P100=0.030834012666786672, P101=0.027201523918474083, P102=0.009781182001184933, P103=0.009407373956330872, P104=0.010388697840307963, P105=0.03223982882679347, P106=0.06670335113161933, P107=0.03831108936318622, P108=0.0408582074428328, P109=0.02236167031235124, P110=0.0261521602749944, P111=0.033010411156739035, P112=0.014036267156750423, P113=0.015923532832898976, P114=0.046964445724693354, P115=0.031373474091880474, P116=0.0313282615657196, P117=0.01953886671730506, P118=0.011079897673513334, P119=0.01602689219257812, P120=0.008477315620779014, P121=0.018024365455933118, P122=0.006935465671771207, P123=0.012659466877228341, P124=0.02262610996416101, P125=0.010960083216530095, P126=0.010171569495428872, P127=0.005530130446827467, P128=0.005251491555212962, P129=0.010465960630300922, P130=0.009480628131316959, P131=0.012871361255752473, P132=0.018415633807628562, P133=0.01303960561339264, P134=0.02541695435562698, P135=0.02017122214176151, P136=0.01748122371011869, P137=0.01429570895354424, P138=0.007167763789232169, P139=0.008648498127678936, P140=0.00871409235677266, P141=0.010007322031489664, P142=0.009702788452348537, P143=0.008758442976313296, P144=0.008946431499056023, P145=0.003522577501377775, P146=0.0026437618975335953, P147=0.0016059229840191603, P148=0.0010008949896926267, P149=0.0006833407210681413, P150=0.00041438336067087776, P151=0.0004774712207292929, P152=0.0008202370952104357, P153=0.000778454038486902, P154=0.0006701967558503697, P155=0.0007335383693257585, P156=0.0009600909024383718, P157=0.0004768839872060957, P158=0.0005907572384862183, P159=0.0014057076905157242, P160=0.0022164704971380533, P161=0.0013853160775482397, P162=0.0007777283475931462, P163=0.0016377689740875048, P164=0.001843489426871514, P165=0.0014038117964588885, P166=0.0008439757071448489, P167=0.0015665956944173, P168=0.0018681023887127885, P169=0.0008746502871458038, P170=0.0007197184579502254, P171=0.0008936191684035867, P172=0.0006899396571371397, P173=0.00037246677130941587, P174=0.00037593167186885073, P175=0.0007128288928820519, P176=0.0008090065149068705, P177=0.0004965490439599922, P178=0.000271498419156711, P179=3.610642101629617e-05),Row( P0=-569.8056495794865, P1=56.73135890438738, P2=-8.105669299032943, P3=7.37694660723386, P4=-1.0517106674098287, P5=-10.465031945267832, P6=1.641751274159077, P7=1.7410795757647062, P8=2.6662215624598997, P9=-3.6727049759889896, P10=-2.6976015834384515, P11=0.6295405976772858, P12=-5.232518283337202, P13=1.3450054973178562, P14=1.0125670380224328, P15=4.935647683664295, P16=8.079808859280947, P17=4.800735460496564, P18=9.21825511771498, P19=9.35470571072938, P20=1.552176246654965, P21=5.741781675015949, P22=2.5940807400730046, P23=2.7028244463747106, P24=3.057455807138066, P25=6.9306128674852925, P26=2.4369675048520825, P27=4.345240257870663, P28=2.331182895252088, P29=4.113442743383309, P30=1.1448605011190105, P31=3.135822804060641, P32=1.487548847324548, P33=2.167922905696983, P34=-2.134618063778532, P35=2.5852120624983064, P36=0.7566915739088176, P37=0.008776132801942775, P38=-2.3171188391595425, P39=-1.2034398887699782, P40=0.6953638061670512, P41=0.6429987071869654, P42=0.6618297588791491, P43=0.6354737811949085, P44=0.6502484565820698, P45=0.6768437258770346, P46=0.6400310926266697, P47=0.5757351378767843, P48=0.6182092872772695, P49=0.6797854336150055, P50=0.7543605223067997, P51=0.7584780116444546, P52=0.002724029625737824, P53=0.0009644587966242637, P54=0.00029045407173853064, P55=0.00012374053302425346, P56=0.00022392518309544165, P57=0.0004505890088952259, P58=0.0014021523307809049, P59=0.006312405569866122, P60=0.03472386428476294, P61=0.12578419143298708, P62=0.21326155664136862, P63=0.22119432887994936, P64=0.1425672707721918, P65=0.18925247098172795, P66=0.27580504243533405, P67=0.21402477168225972, P68=0.09102445680691812, P69=0.05921932533316054, P70=0.03334258167839082, P71=0.025932309893117282, P72=0.04799438361473536, P73=0.06723997006563232, P74=0.025930219228193156, P75=0.015986032142546045, P76=0.023485241551412437, P77=0.04898013335099268, P78=0.06202783730884287, P79=0.08033407179156529, P80=0.09601886648503649, P81=0.11559241274937762, P82=0.11042236528778579, P83=0.1539710305390823, P84=0.24181958444311136, P85=0.11336063727388951, P86=0.03322173696028957, P87=0.030466681726156016, P88=0.052278242595682214, P89=0.03078112700781837, P90=0.014051432019567024, P91=0.01641357557837329, P92=0.03905916683493759, P93=0.049488800587324515, P94=0.05059031551786658, P95=0.03544446529015732, P96=0.013265905281560863, P97=0.011085403922376018, P98=0.010211977527174858, P99=0.00850374745643666, P100=0.00516298289236697, P101=0.005206612706097368, P102=0.007846138575967488, P103=0.006001341156781283, P104=0.006203343665967444, P105=0.007288035997060446, P106=0.010972730025787657, P107=0.015337888188616203, P108=0.028315552570520028, P109=0.018308010033377904, P110=0.005117129295413613, P111=0.003363637251151516, P112=0.0017276676958243283, P113=0.001737544136117542, P114=0.007378855004735305, P115=0.006250275217160346, P116=0.0029539170431291586, P117=0.0019520113718917338, P118=0.0013133360659830292, P119=0.002522012088382267, P120=0.0044489605674656344, P121=0.003600461703791498, P122=0.0031240834648396166, P123=0.0030421685978055685, P124=0.003914122098054353, P125=0.0038747464410504016, P126=0.0021940802256286977, P127=0.0013178574257739688, P128=0.001271691289206223, P129=0.0026468631171011206, P130=0.0033562815239487854, P131=0.004366972348633991, P132=0.004334865301466186, P133=0.0052299466422492885, P134=0.006864667914739392, P135=0.012558673319261802, P136=0.006218918515066035, P137=0.002744249987533215, P138=0.0014797880220159118, P139=0.0009935187084716168, P140=0.0013912999953241986, P141=0.0008676708239458543, P142=0.000883180063465698, P143=0.0005636286265106984, P144=0.0012734679737750893, P145=0.0017633572463710623, P146=0.0018612434449433147, P147=0.002076368117344672, P148=0.0008589656607146091, P149=0.0005191399931404505, P150=0.0006195406872943812, P151=0.0005861297977537542, P152=0.0005319363471305886, P153=0.0008311619488017217, P154=0.00076034833585355, P155=0.0004934729913534037, P156=0.0006365647260133979, P157=0.0007629133548343434, P158=0.0010522449217704372, P159=0.000654504888445566, P160=0.0007132873030271554, P161=0.0007398117286054468, P162=0.00043787040396449964, P163=0.0004093647740193816, P164=0.00033117411542874127, P165=0.00032081600096500615, P166=0.00030023651160200843, P167=0.00027570210438248905, P168=0.0003163320637155467, P169=0.0004199348034272506, P170=0.0003212869470515394, P171=0.00027805870813087677, P172=0.0003009501708009878, P173=0.00037185400234996475, P174=0.00036419307508012247, P175=0.000407039444468478, P176=0.0005038696851242211, P177=0.0005514367420599297, P178=0.0004012829867669761, P179=5.3696216987086606e-05),Row( P0=-551.5547155212674, P1=58.47224742339989, P2=-17.957106439380837, P3=7.6277526222565655, P4=-0.6832590065656079, P5=-8.89693710242193, P6=3.28668195033006, P7=3.9143217734447555, P8=5.798645821571127, P9=0.34954749813636676, P10=0.12676510692585052, P11=2.018697751351762, P12=0.017068336734725034, P13=3.0426984196000437, P14=1.327457907529823, P15=1.7295615994387956, P16=3.9055190357651663, P17=0.35374805869440645, P18=3.5899570289890717, P19=5.558673898543529, P20=3.0878548056489525, P21=4.509333900691396, P22=5.512532444904599, P23=3.3615308030920485, P24=1.5588434695915065, P25=3.020005751222711, P26=-0.20512160685304698, P27=2.2769598630406676, P28=0.023352933482873543, P29=3.365323202348704, P30=1.9966474304250212, P31=1.491035518574334, P32=0.038782547282415404, P33=3.231838606575353, P34=0.9687068130869951, P35=6.044807858409709, P36=3.8730024079857492, P37=4.9931778055074645, P38=1.4074000917004759, P39=1.1922690512530218, P40=0.6354520285125872, P41=0.6817732527700809, P42=0.6980099976345532, P43=0.6996207722175926, P44=0.7193547688916468, P45=0.7553182002135149, P46=0.7399678392924808, P47=0.6527601441151344, P48=0.6559458306112411, P49=0.682966480204753, P50=0.6859485825893211, P51=0.6577508818226508, P52=0.9191271161328952, P53=0.17284674093788044, P54=0.005713878984917732, P55=0.00012497209208538673, P56=7.805093468643807e-05, P57=0.0012484735198315985, P58=0.010938195135018564, P59=0.023783563033824277, P60=0.03736708273393802, P61=0.05331794858239401, P62=0.14752732972231997, P63=0.23159549206128827, P64=0.0617643917349319, P65=0.021198743335638308, P66=0.030684863880169843, P67=0.026606615633866636, P68=0.01922562698385648, P69=0.029411552566120852, P70=0.03425029472264336, P71=0.012227066723007808, P72=0.0101454119799717, P73=0.016145449107920987, P74=0.03410956642370245, P75=0.04984912603849951, P76=0.031942328049075944, P77=0.023202814040684847, P78=0.02836306592317229, P79=0.042955008738806935, P80=0.05872315573991048, P81=0.05002838171453243, P82=0.03368289618317784, P83=0.026954774014615244, P84=0.033328700042396235, P85=0.05445060342069439, P86=0.08264841678992675, P87=0.08284717268633575, P88=0.09205956733853173, P89=0.07868000041280218, P90=0.05076231436484285, P91=0.025844309952755586, P92=0.04224111309268802, P93=0.023542171197661586, P94=0.1198648137580243, P95=0.13741340733154733, P96=0.03332525022344753, P97=0.00735416506760314, P98=0.006780544120469875, P99=0.002333780575746578, P100=0.004315445987965013, P101=0.007636907212637609, P102=0.0036437373187326824, P103=0.0021453871980756547, P104=0.00508827271671017, P105=0.003263543099969864, P106=0.0066748723040178405, P107=0.00740242688982405, P108=0.009369654529552877, P109=0.0038329116610566058, P110=0.010557219055954496, P111=0.04621009328170489, P112=0.014534381318592046, P113=0.007637891537671762, P114=0.014212253568888163, P115=0.007473951506366746, P116=0.024384437726742057, P117=0.012586386623419835, P118=0.007634067975391742, P119=0.011527687225607269, P120=0.01271008846979126, P121=0.018909501482676113, P122=0.004728652379367621, P123=0.0048031706976180855, P124=0.0038846914606548667, P125=0.005172105724970551, P126=0.0022619159135678587, P127=0.0022577148440953115, P128=0.0037148290414869274, P129=0.0037773299708600996, P130=0.006645342631910795, P131=0.004345426078214869, P132=0.005302685443109519, P133=0.0030941130133643646, P134=0.002719848296517548, P135=0.002959853556539527, P136=0.001160940472058626, P137=0.001213089181116091, P138=0.0008416388558758746, P139=0.0007081970376340406, P140=0.0006986300418563863, P141=0.0006026146664358965, P142=0.0009084256965229982, P143=0.0007688959946770587, P144=0.0007268293890592603, P145=0.0005181572114477763, P146=0.00048411211659251085, P147=0.0006680076371265706, P148=0.00035184236637220525, P149=0.0001788226991216907, P150=0.00015370832947925967, P151=0.00014130473736054865, P152=0.0001916005230304448, P153=0.00030356994132603465, P154=0.00033376520739692643, P155=0.000473780341851154, P156=0.0004366605730271797, P157=0.0002965238522275297, P158=0.00021560569183192132, P159=0.00014342404714439688, P160=0.00010089403870545902, P161=9.716848601563487e-05, P162=0.00011682887338655088, P163=0.00014268711674513435, P164=0.00015249726057308617, P165=0.00015917062691780603, P166=9.759809691635122e-05, P167=5.702174904389993e-05, P168=8.473467390574447e-05, P169=9.443903839451482e-05, P170=0.0001044768977951002, P171=0.0001066405706183627, P172=7.842635252029828e-05, P173=8.49757284379715e-05, P174=0.00010602250382549147, P175=0.00011609914359619673, P176=0.00015654990939095346, P177=0.00020116004906479268, P178=0.00011286592079709195, P179=1.1490852816379542e-05),Row( P0=-620.1496711171433, P1=68.79532826750115, P2=-12.2776555646076, P3=13.520298839948241, P4=-2.5557667334232606, P5=-10.414260900938265, P6=3.66476589620458, P7=3.3080058452477297, P8=3.1518369281941836, P9=6.307703807762822, P10=-0.230850111333139, P11=0.5576300436693694, P12=0.14830530362479236, P13=1.3351333355258992, P14=-1.9538350817656565, P15=1.7445331004251896, P16=2.19136294772233, P17=1.0428662606657315, P18=2.1693962618657974, P19=-0.2137392508018722, P20=1.572931108635846, P21=-0.3846321102798742, P22=2.117861125437622, P23=0.6774692819708669, P24=1.5041620409649132, P25=3.730128699453766, P26=2.5513069015490375, P27=3.9974327613019756, P28=1.920800317799801, P29=4.091707902012406, P30=1.0646810965127647, P31=3.1862521941170154, P32=1.320277114682275, P33=4.165281170114009, P34=3.3989694136977597, P35=5.762927962896773, P36=1.9200880790510402, P37=1.2400770331948514, P38=0.06445928357466005, P39=1.713675753718248, P40=0.6914128850455141, P41=0.6791160199446816, P42=0.6926305263583793, P43=0.7437983022430152, P44=0.7342307296240493, P45=0.7074399413408631, P46=0.7082891121698222, P47=0.6723949830350983, P48=0.6381757749860183, P49=0.6740186457946585, P50=0.7058611593703983, P51=0.7173837397072045, P52=0.015894833360503374, P53=0.0021397337770063265, P54=0.00030549918484394704, P55=6.465362217377235e-05, P56=9.776386815353549e-05, P57=0.0010937830044590237, P58=0.008270885671021503, P59=0.023794159864051416, P60=0.055862533974118694, P61=0.08977496885863562, P62=0.09741131840242909, P63=0.06257393197955932, P64=0.012641829646061933, P65=0.009446837924875967, P66=0.021141029774573352, P67=0.02772868604953317, P68=0.02265022826105369, P69=0.051143464965158696, P70=0.08486339374015628, P71=0.04631253746859343, P72=0.023882529925248516, P73=0.022486099029214356, P74=0.03165587045306707, P75=0.03560010036560236, P76=0.015588449135550492, P77=0.009580037208312095, P78=0.014049022032239438, P79=0.023777982078037954, P80=0.03675090602930838, P81=0.049302485581132025, P82=0.0765389421488884, P83=0.09266045410018245, P84=0.10235483588352189, P85=0.06390453956388968, P86=0.023331781043657275, P87=0.01892363328624966, P88=0.061022284032324965, P89=0.06637837779535403, P90=0.05069842569841055, P91=0.06546270152834789, P92=0.03490643929312062, P93=0.011305583692387926, P94=0.010761143958260777, P95=0.08983940469545379, P96=0.06899893168436205, P97=0.03004490314054995, P98=0.005097154086292487, P99=0.0014285456952573322, P100=0.0029988693508861066, P101=0.00687372327970742, P102=0.005105424347266209, P103=0.0072744517919675076, P104=0.002261073148846949, P105=0.0053227083941797424, P106=0.008355458714278933, P107=0.0025916410704693374, P108=0.0031128405933467554, P109=0.013102434235884598, P110=0.026315920193653416, P111=0.014630003785543686, P112=0.0053824606256135145, P113=0.01463482511675807, P114=0.007931281639587178, P115=0.008091332874668226, P116=0.012239707165091884, P117=0.004540803992380477, P118=0.003581689109999592, P119=0.0033748318027734935, P120=0.005582422911542592, P121=0.0033746336477450648, P122=0.0031732706241124394, P123=0.0016537496574778306, P124=0.0038271822227335006, P125=0.0028775895959642405, P126=0.001935421960263897, P127=0.0041662045447732055, P128=0.0022096215448714294, P129=0.002573219172859087, P130=0.0023102484970115946, P131=0.00468647426558682, P132=0.004132115160298192, P133=0.00790253808803161, P134=0.004896542400187337, P135=0.003194031179934889, P136=0.001978767168161188, P137=0.0013551912991168087, P138=0.0014108230947237448, P139=0.00145446267889262, P140=0.0011413111900424825, P141=0.0010719773411741206, P142=0.0006187637164589701, P143=0.0004326701194623703, P144=0.0003480453674835453, P145=0.000311100175024491, P146=0.000198945270522538, P147=0.00013504737761661537, P148=0.00013076779825821756, P149=0.0001143042243293492, P150=0.00018949173556253055, P151=9.058337383379318e-05, P152=8.015518869191513e-05, P153=0.00016468887206395208, P154=0.0001609625095452001, P155=9.115360022524453e-05, P156=0.00010593557758953544, P157=0.00023430652457545977, P158=0.0003882440455325708, P159=0.0002313097223143103, P160=0.0001137673037904458, P161=5.933713157417736e-05, P162=0.00011756524463004334, P163=0.00012614055124192714, P164=8.634218963827788e-05, P165=0.00012804242583249205, P166=0.00015849582636941033, P167=0.00015844497722734585, P168=0.00015885717375586076, P169=0.0001142554757686442, P170=7.324416159972093e-05, P171=8.180532894714511e-05, P172=4.5379914886432245e-05, P173=4.3343729771346565e-05, P174=5.982271318330791e-05, P175=7.2794690921309e-05, P176=6.060799422251933e-05, P177=6.740017774031694e-05, P178=4.9386617242783663e-05, P179=7.0378474520806015e-06),Row( P0=-566.7985397074392, P1=49.77273976759597, P2=-13.653394228580842, P3=2.120883242355887, P4=-13.585803133732457, P5=-16.03865867085254, P6=-6.547781918067811, P7=1.8218439489198073, P8=1.518604537432923, P9=-4.109808030106101, P10=-2.3726921697801107, P11=4.039127635634221, P12=-3.6882723050920707, P13=-2.54480744462789, P14=0.3304242757481662, P15=3.715083374047848, P16=3.956077383329929, P17=4.084139266883644, P18=6.302620123206643, P19=7.420800139891287, P20=6.936381857258247, P21=7.530443717381575, P22=4.82885185490306, P23=4.46115745931923, P24=4.397675435502345, P25=8.176282412791256, P26=2.599600916332873, P27=6.55784778549344, P28=2.7662244657619652, P29=8.20142728728172, P30=2.1399190283545155, P31=4.999173624585003, P32=1.0591936540139486, P33=4.4786655517031875, P34=-0.16130335947794855, P35=6.742728910199829, P36=1.515617684623004, P37=0.9224835602870884, P38=-1.2558028455491552, P39=1.6841326141454165, P40=0.7128103414621326, P41=0.6014806113081751, P42=0.5941468432120581, P43=0.656236913918925, P44=0.6536950913374371, P45=0.671465668699017, P46=0.6907377406896937, P47=0.7170096298688591, P48=0.6638299890843812, P49=0.6919122996422595, P50=0.7539165177492633, P51=0.7869685592935057, P52=0.01856718708028251, P53=0.0022953170499529258, P54=0.00012318373356535764, P55=3.3912787090734646e-05, P56=7.493796667237177e-05, P57=0.00021793697682038078, P58=0.0010803731222442032, P59=0.004541992672110148, P60=0.017737942097903428, P61=0.07544120689001808, P62=0.09115086624324231, P63=0.09175943392785701, P64=0.04912333663788627, P65=0.07635328178310667, P66=0.10393349660717906, P67=0.10146243311712275, P68=0.030104292057911135, P69=0.00905126803787004, P70=0.014269640571376242, P71=0.021890679123688254, P72=0.035415282984172916, P73=0.025374311163191755, P74=0.030275154920513168, P75=0.04665157112101878, P76=0.022874336264480613, P77=0.01762941486590184, P78=0.031199774966009414, P79=0.05999531570158906, P80=0.06930789969594763, P81=0.062441630242449836, P82=0.09080891035081612, P83=0.0827023823509757, P84=0.04858977705053567, P85=0.04772809196210699, P86=0.0395312170818532, P87=0.024608112131689303, P88=0.03845021024387703, P89=0.03052801376009689, P90=0.016121566329388666, P91=0.009941508039002648, P92=0.008624877131849664, P93=0.0039572629592000315, P94=0.025680698813398368, P95=0.07585289597882756, P96=0.027992936538913413, P97=0.003957223261009626, P98=0.005256658172467896, P99=0.0024830886492438592, P100=0.002712506035036883, P101=0.003997032390372602, P102=0.0035671687300349642, P103=0.003392999817143499, P104=0.0017903918449803094, P105=0.0021586455304129586, P106=0.0026990569345807626, P107=0.0018538164139466264, P108=0.003193798113206923, P109=0.003599980883564491, P110=0.0029166042733884423, P111=0.003865970524226794, P112=0.004388862457181163, P113=0.006903841749610503, P114=0.001871183904258159, P115=0.0012189408152711724, P116=0.005365825566252976, P117=0.008423683647945459, P118=0.000792048593404022, P119=0.001913439310372247, P120=0.016213514427069378, P121=0.006142829816215835, P122=0.003891532983165304, P123=0.004522148372413318, P124=0.0023837604961693425, P125=0.001402751270975689, P126=0.0020413560980112886, P127=0.0006551449774074933, P128=0.0007582782816191965, P129=0.0020962864421486638, P130=0.0004326891100511462, P131=0.0006775214950763229, P132=0.0006303855281393806, P133=0.0008393984968310744, P134=0.0016956114399724732, P135=0.0023392699509137663, P136=0.000614197200593013, P137=0.0007609707362615382, P138=0.0006005829247470231, P139=0.0004474886770691942, P140=0.00025629210438634503, P141=0.00016513412518682547, P142=0.00016769124823152903, P143=0.00026432476072619756, P144=0.00033302093498292827, P145=0.00032039695196252094, P146=0.00038921298459544527, P147=0.0002846635372252884, P148=0.0002693537774040724, P149=0.00016722171439863463, P150=0.00010117130180425957, P151=0.0001326357897966801, P152=0.00030041856093674535, P153=0.0004179775046130438, P154=0.00044290935599062767, P155=0.00038551384766661424, P156=0.00038608670011087217, P157=0.0003783288877084703, P158=0.00033869549669612295, P159=0.00026748226793855127, P160=0.0003241922965771779, P161=0.00024020628139944977, P162=0.00022769080107936548, P163=0.00016629165857702698, P164=0.0001090784701868248, P165=8.982769729025461e-05, P166=0.00011402333459929527, P167=0.0001441222086014028, P168=0.00018470855624686753, P169=0.00017406349114427346, P170=0.00013431150692484904, P171=0.0001244037560403184, P172=0.00014847201033388145, P173=0.00015407709208102685, P174=0.0001527354864417952, P175=0.00030937977175481065, P176=0.0003420829191365922, P177=0.00046571225391154507, P178=0.0002612804110994057, P179=2.7701838972301107e-05),Row( P0=-476.55705568365033, P1=35.022063258330014, P2=-7.579826853349684, P3=8.139199372417476, P4=-7.426711533332665, P5=-8.528094169773066, P6=0.41186745650791445, P7=0.7220122426800674, P8=-0.2672489568658261, P9=2.7086279356602243, P10=-1.8560049031014534, P11=4.6302875342704475, P12=0.8012816225989998, P13=4.553911064873519, P14=4.031531732743818, P15=6.663148607036748, P16=10.469572488381885, P17=11.17668803609595, P18=13.21035210875386, P19=13.664571831104476, P20=9.066340962632767, P21=5.973754974248808, P22=1.7065535282905442, P23=1.6886859487661858, P24=-0.572256644107147, P25=4.901250455568118, P26=1.4582088864703604, P27=3.7310499525143626, P28=0.9215390564898718, P29=0.7030249256737354, P30=0.4373968856595734, P31=3.822411760902813, P32=1.1557039606876218, P33=1.9792558553695372, P34=-2.206200525465633, P35=2.867477475034209, P36=1.7453881246119034, P37=2.7478088115655943, P38=0.5979537692475848, P39=0.5426531268759885, P40=0.6473036547558706, P41=0.6181747343672044, P42=0.6195364913424801, P43=0.6441902812559864, P44=0.6349157054191868, P45=0.657244681343458, P46=0.656761655383636, P47=0.6703424847323313, P48=0.6731975832653228, P49=0.7526516920216164, P50=0.7893765185140962, P51=0.73697801911676, P52=6.828009816090573, P53=1.1145070066969145, P54=0.059595215018832956, P55=0.003108803365016781, P56=0.000876657357906387, P57=0.0004921797325380616, P58=0.00022563337115450195, P59=0.0009124773914823823, P60=0.004867996328848689, P61=0.015596626908308769, P62=0.049713630071494715, P63=0.1221888561235937, P64=0.32843246101986573, P65=1.7854662893719107, P66=1.5509951954709793, P67=0.5216589842262387, P68=0.2728977467115572, P69=0.2740472985835732, P70=0.29060569011453863, P71=0.08763936638887145, P72=0.01637216069608789, P73=0.022146389427584453, P74=0.06221984495742665, P75=0.11257993630339065, P76=0.08288620860617826, P77=0.12793202697187786, P78=0.5212265717233988, P79=1.5750658278752205, P80=2.6018012651474196, P81=1.1626682011336926, P82=0.27488958131350066, P83=0.12043092877348803, P84=0.12421147737412008, P85=0.04261418658948937, P86=0.011478241114779185, P87=0.037565483886519056, P88=0.09068803305396932, P89=0.0433272564474141, P90=0.11554531698296301, P91=0.5043424678738706, P92=1.3561895729644913, P93=0.7797651537903463, P94=0.0597796327448141, P95=0.037944293737442014, P96=0.07067862671607879, P97=0.05123957505211206, P98=0.013564995982541477, P99=0.014472627358413774, P100=0.06557246952464278, P101=0.1315108097882738, P102=0.09477619138751688, P103=0.012980038916678068, P104=0.0035044202852737532, P105=0.012543358305084995, P106=0.016884253550261713, P107=0.0504298970766012, P108=0.14358303451700968, P109=0.06948670330993353, P110=0.004712741732048807, P111=0.01668196984036183, P112=0.04755491702626487, P113=0.07284657996020961, P114=0.013388949586969637, P115=0.003931179041406328, P116=0.009197005374895834, P117=0.03920239021738343, P118=0.019122545289303777, P119=0.0075363525325417265, P120=0.007591351366621879, P121=0.012258229285936298, P122=0.011211391297312506, P123=0.00849889487141768, P124=0.009245288230556077, P125=0.009343201709702263, P126=0.005670012614462616, P127=0.007941404344672493, P128=0.0051932587105470406, P129=0.0076577744057230125, P130=0.006776000815468772, P131=0.010214658665500751, P132=0.008770972630152285, P133=0.007420550264564021, P134=0.01068408049582949, P135=0.01313749667257352, P136=0.007685495264174924, P137=0.005324831210905877, P138=0.0026250852819474715, P139=0.0019123955002242087, P140=0.00202440610581772, P141=0.00190416295401729, P142=0.0027081221812624355, P143=0.0026329730369307413, P144=0.004188889074713677, P145=0.0027339963228994696, P146=0.0028728875093368958, P147=0.001754679893633666, P148=0.002713132457277043, P149=0.0018350437247188437, P150=0.0009868011048584635, P151=0.0009102870324146286, P152=0.0011748191573517541, P153=0.00108586937183831, P154=0.000788960672314718, P155=0.0007495055844691343, P156=0.0006697935618895209, P157=0.00047225067727194573, P158=0.00045162776281345093, P159=0.0005002099795497978, P160=0.0004996974722154858, P161=0.000556887127515719, P162=0.0006911173594167425, P163=0.00045153634093244714, P164=0.0003405199354182421, P165=0.00044731483850544355, P166=0.00056182950417888, P167=0.0006055513599528806, P168=0.00047548607882388406, P169=0.00031819149484070563, P170=0.0002192491586834279, P171=0.000161244594539603, P172=0.00016413404697725894, P173=0.00028028713257288496, P174=0.00030800943299274326, P175=0.00024495104740351696, P176=0.0002670262602856079, P177=0.00024108675522697422, P178=0.0001037148298796294, P179=1.1016304786590522e-05),Row( P0=-563.39882017927, P1=63.292424410117334, P2=-15.957634258202539, P3=18.227166018878705, P4=-2.0650607644283574, P5=-8.357061874166263, P6=7.176613291331381, P7=1.6887493746063844, P8=2.1429673731594807, P9=5.579301999283952, P10=-4.494461683022232, P11=2.6138440066520836, P12=1.5311898004841142, P13=-0.22208454140225786, P14=-0.5046258528154772, P15=1.659971562709353, P16=-0.3198767909703629, P17=-0.05364005856016035, P18=1.3984699280828643, P19=0.8253904568557686, P20=2.5783197751510714, P21=0.7009919404746938, P22=4.023142234790356, P23=3.003888701591314, P24=0.9631144261411957, P25=2.656962912423457, P26=1.0844627581798267, P27=3.145341297420746, P28=1.2397095673108889, P29=4.035752500648426, P30=0.857980895743309, P31=3.76720175896788, P32=2.5641780423934346, P33=5.25220409262995, P34=2.5745932227040913, P35=4.519597096304898, P36=-0.05135433244698997, P37=1.909818510838334, P38=2.3622143967577007, P39=2.6529641456794324, P40=0.6764170824762643, P41=0.6393836108724343, P42=0.6483243602206943, P43=0.6827192882199773, P44=0.7192449140573496, P45=0.7102762273604382, P46=0.7422762258921937, P47=0.7195538145449312, P48=0.7503783196100829, P49=0.683555462516468, P50=0.6578866138479704, P51=0.6808162380880365, P52=0.28468215651798856, P53=0.016762101559021764, P54=0.0006726333428105409, P55=0.00015628807022436935, P56=0.0002851853306038254, P57=0.002233673555581899, P58=0.00961422319312526, P59=0.024372297055328604, P60=0.05410503061861444, P61=0.16126477257380648, P62=0.1676307585104815, P63=0.11292242077021421, P64=0.06550225283601491, P65=0.022106058700530395, P66=0.02678069115394605, P67=0.04118647050781786, P68=0.03653400058002672, P69=0.04885635228861356, P70=0.06131699026995471, P71=0.07631575929628141, P72=0.06403119229449654, P73=0.029584411681910214, P74=0.03627560370449843, P75=0.05642801311317258, P76=0.16471358844273656, P77=0.25497724362110125, P78=0.08433056405651296, P79=0.046622753812203516, P80=0.03188756452397777, P81=0.03727773764021613, P82=0.06231934469873395, P83=0.06606473404067831, P84=0.13109671287011962, P85=0.13327746257718837, P86=0.04289783162705478, P87=0.051412468457355316, P88=0.1323951169918908, P89=0.08153576898862443, P90=0.05497586697785876, P91=0.19220967559090232, P92=0.18490934517162957, P93=0.08064318123413776, P94=0.17143665352829654, P95=0.10841664504640375, P96=0.03337651861545466, P97=0.08532332216454842, P98=0.36798712599496897, P99=0.12806658770343374, P100=0.01652351296986079, P101=0.00625354230063888, P102=0.003854455660033808, P103=0.011161939186962878, P104=0.0180157146252284, P105=0.009259653109834051, P106=0.006831098425961182, P107=0.0041771575632302054, P108=0.04577917221922269, P109=0.03651982464174117, P110=0.01321313196247031, P111=0.02414264929028236, P112=0.038282199262182785, P113=0.03731239068585748, P114=0.014413121839549478, P115=0.018581228467936445, P116=0.0327692621170577, P117=0.009785467488684295, P118=0.02174754907171437, P119=0.020513798909838784, P120=0.01625806469527708, P121=0.012927864152108377, P122=0.01481259447176878, P123=0.010289485880597107, P124=0.011450471800747769, P125=0.011514782504521924, P126=0.003668890519109386, P127=0.00708520483670744, P128=0.005874004825877689, P129=0.004518255700496126, P130=0.0052662700744407764, P131=0.00893806606526064, P132=0.01198833285773721, P133=0.024331166342375983, P134=0.02409577036621463, P135=0.015426031956850406, P136=0.010057492738772296, P137=0.00739996346768211, P138=0.007437812903947141, P139=0.01092866769052896, P140=0.011453699927645677, P141=0.013057820828518661, P142=0.007025689387995746, P143=0.003231082627649894, P144=0.002297055157010258, P145=0.002427582128532703, P146=0.0016114625257959792, P147=0.0008011525552526103, P148=0.0003660797356723261, P149=0.0001379832920077161, P150=0.000124286728888016, P151=0.0001227304400860867, P152=0.00017871696087191864, P153=0.00017974759055922027, P154=0.00029663073799965844, P155=0.0003518057587028936, P156=0.0003792283462125478, P157=0.00026213839094167945, P158=0.00022892235116350138, P159=0.0001764319644073909, P160=9.401434464241774e-05, P161=8.650204880157592e-05, P162=7.924122793761257e-05, P163=5.708434643829532e-05, P164=8.833227160786935e-05, P165=7.202963500417946e-05, P166=8.605750212906828e-05, P167=8.661167918042094e-05, P168=7.995040869947585e-05, P169=7.36718060437655e-05, P170=4.2227754996913906e-05, P171=3.6858180359020625e-05, P172=2.5225972462330056e-05, P173=2.369268463325105e-05, P174=2.7338662235723356e-05, P175=3.779271736870649e-05, P176=3.604401854358358e-05, P177=3.266587176554786e-05, P178=1.6234052847773986e-05, P179=1.647608569443843e-06),Row( P0=-635.8822786440094, P1=91.85317451238875, P2=0.8739031600009947, P3=11.413980210763517, P4=-3.707688553920287, P5=-3.637349382218823, P6=1.6193048677120376, P7=-1.333934197782897, P8=3.3991419147293755, P9=2.835958064682528, P10=1.2479976017775618, P11=-1.095338588122946, P12=-0.7805493095631251, P13=1.909016381230418, P14=-3.692049318694523, P15=0.8514381339257192, P16=0.3140880831916353, P17=-0.6610585096314268, P18=3.2769179060197318, P19=0.5250052955264636, P20=0.6781708857922462, P21=-0.9796340315371059, P22=2.1356306527333375, P23=0.43986020489876326, P24=0.6684067777733252, P25=4.691203810261206, P26=2.7816200227588697, P27=8.797427087786735, P28=5.396059411983292, P29=8.906084612859877, P30=6.042892652907414, P31=7.56796227958648, P32=3.602722541862138, P33=4.35520955167079, P34=0.3535062291836193, P35=1.5959316149514493, P36=-2.2702104260857054, P37=-1.0710242164637063, P38=-0.6850709022122302, P39=1.081872213520549, P40=0.6881604325701987, P41=0.6861604032934239, P42=0.7224168706042949, P43=0.6974880101698714, P44=0.6704350958826018, P45=0.5987473285019894, P46=0.5805347466841371, P47=0.6191892564391589, P48=0.6159976007587522, P49=0.5926616349454406, P50=0.6647882471147978, P51=0.6829932942126197, P52=0.001176778596767531, P53=0.0004412580606965314, P54=0.00016765372005727078, P55=2.8862395826016586e-05, P56=4.0770904200995474e-05, P57=0.00018796839359163658, P58=0.0016741138567589805, P59=0.016684810721026, P60=0.050235079360943485, P61=0.06674542928698324, P62=0.04228359292625765, P63=0.0064912229328360615, P64=0.000968156963665991, P65=0.0010005217699165761, P66=0.0025209716945329085, P67=0.005979690559056226, P68=0.027341824005032177, P69=0.03457327214485829, P70=0.028192187732202095, P71=0.023763623826325967, P72=0.021119482731791556, P73=0.010858233146250638, P74=0.0030571623361134326, P75=0.001856321774010976, P76=0.006335333178227041, P77=0.019055295268065223, P78=0.014933915978890867, P79=0.012412152521905025, P80=0.017008817039374208, P81=0.04541122411232152, P82=0.054674420673621896, P83=0.019863175998537798, P84=0.012958032419548305, P85=0.016764377875414044, P86=0.007944765212663175, P87=0.006701644557149548, P88=0.010893430098184598, P89=0.010100479691464425, P90=0.015275903672228792, P91=0.03333401361817954, P92=0.031066577747269144, P93=0.008301635475510293, P94=0.00919457094982103, P95=0.010850495168609973, P96=0.00838240768275349, P97=0.011810874303423686, P98=0.012769856318111056, P99=0.004064154163332104, P100=0.0019728487836980935, P101=0.001942594089361822, P102=0.0024990054536856987, P103=0.004283751890420482, P104=0.0038697659101268374, P105=0.0037665269525570216, P106=0.0043145774780305336, P107=0.0022320740795535566, P108=0.004113590460243814, P109=0.006269446492928727, P110=0.00503431935637382, P111=0.0013550873027116893, P112=0.000648617477613266, P113=0.001548819408215026, P114=0.0010304439722857352, P115=0.0003505044033637629, P116=0.001100309931053244, P117=0.00039168972421895257, P118=0.0002630985092910551, P119=0.00044500399924862473, P120=0.00023341538184846555, P121=0.0002662928554158789, P122=0.00015328765006854036, P123=0.00021322969269054788, P124=0.000535554893824276, P125=0.00012968849992489007, P126=0.0002529419201846046, P127=0.00021179625190644713, P128=0.0001319019019318903, P129=0.0001234012299472292, P130=7.856960915758e-05, P131=0.0001659436196580634, P132=0.00016212324405035032, P133=0.00033075771741694583, P134=0.00032437802473028754, P135=0.0003635022145144871, P136=0.00026615663035504597, P137=0.00014303352211767276, P138=9.670602135734018e-05, P139=6.294710944627772e-05, P140=5.152468066592173e-05, P141=6.132752408905672e-05, P142=6.826653694495483e-05, P143=7.587973928613927e-05, P144=6.047114673161619e-05, P145=5.16755386491682e-05, P146=6.750884169818895e-05, P147=6.41168648426707e-05, P148=8.625273585351846e-05, P149=8.202048236484198e-05, P150=4.2072732651107364e-05, P151=3.94316226457313e-05, P152=6.043911325423225e-05, P153=0.00011327010264685792, P154=0.00013457114729550107, P155=0.00014981074176046653, P156=0.00015114042112173326, P157=0.0001009438964025085, P158=6.193916514163984e-05, P159=5.017064002706413e-05, P160=3.61768100920693e-05, P161=3.40683632929914e-05, P162=5.0188204505065085e-05, P163=5.659917610160181e-05, P164=3.5349196281293305e-05, P165=2.5801355480560623e-05, P166=3.7193672989361485e-05, P167=4.0846111111678365e-05, P168=2.682096787201411e-05, P169=2.3717090300549702e-05, P170=2.0108252924282176e-05, P171=1.0475628217511801e-05, P172=8.594170445461486e-06, P173=9.128302547330382e-06, P174=1.2603228613934777e-05, P175=2.2280674958259514e-05, P176=2.014684014851149e-05, P177=2.2629270978902884e-05, P178=1.0671454331181051e-05, P179=1.645623632563994e-06),Row( P0=-608.4030753170118, P1=59.95437559868535, P2=2.5186459601116478, P3=13.866786696326754, P4=6.02775232771609, P5=-2.5636307474533218, P6=6.225942134536705, P7=6.468868164478208, P8=8.260290438513103, P9=4.263900500018932, P10=2.3695277972059676, P11=5.189489006119364, P12=2.6755450113406867, P13=4.99067921768223, P14=2.0482117287269515, P15=3.845299668550132, P16=5.026815006738294, P17=1.9900060748035446, P18=4.255971454632779, P19=4.500829399745395, P20=2.263885754484395, P21=1.6493718484865831, P22=2.9340727546087235, P23=1.8548777488123334, P24=-0.5323851708666365, P25=3.4691458315418924, P26=1.2685347344652431, P27=5.147191685403059, P28=4.895649212361037, P29=8.102690931051372, P30=6.957457663337422, P31=6.949627622719136, P32=2.049866163924286, P33=3.750606951908603, P34=-0.3497234760874184, P35=2.1800472685744188, P36=0.05099091549386482, P37=0.6096086109337084, P38=-0.08783648554681127, P39=0.10616218521548984, P40=0.7237495152933638, P41=0.7617898810716658, P42=0.7658029495416188, P43=0.7367370948050123, P44=0.706700071039887, P45=0.6976807564995657, P46=0.7073340521684996, P47=0.687050167517248, P48=0.7236203628483047, P49=0.734173704309306, P50=0.7077767110790624, P51=0.7088071807929585, P52=0.659821128679911, P53=0.10187187513470644, P54=0.00345255087986716, P55=0.0001775412103977499, P56=0.00013410080838589791, P57=0.0002543271999735785, P58=0.0013072892960685248, P59=0.017299508233450868, P60=0.053886008472919855, P61=0.048650265351276534, P62=0.02760837299510359, P63=0.018129679933244602, P64=0.003977080647870432, P65=0.0024124138517972677, P66=0.007284114156534986, P67=0.013656894062690621, P68=0.025139332350810954, P69=0.03410240068855289, P70=0.03168818189768937, P71=0.020827739879254435, P72=0.006425008826237325, P73=0.003922188066442004, P74=0.00960641458641905, P75=0.013146679990461756, P76=0.007405097275136917, P77=0.016191970198870802, P78=0.01790951248870446, P79=0.010500365326775092, P80=0.007389829151224812, P81=0.004160202675727688, P82=0.0032301959675805555, P83=0.003398577463931148, P84=0.0068231726323284845, P85=0.021593553159875656, P86=0.03902806437260247, P87=0.018035745994240213, P88=0.004023011731736011, P89=0.003499905170341039, P90=0.003978620329139909, P91=0.007667461911825269, P92=0.04514660207677645, P93=0.02051394121403557, P94=0.003533113037132473, P95=0.0028494816985576996, P96=0.0009350708693036139, P97=0.003729217658952159, P98=0.01089229259161719, P99=0.0034360554928290483, P100=0.000923224989730635, P101=0.0010451619690633454, P102=0.0008293973181586118, P103=0.002335974657202468, P104=0.0006523046540849082, P105=0.0007317200714607447, P106=0.001091125456330866, P107=0.0011246779653330831, P108=0.00045897254400167344, P109=0.00042866816305766703, P110=0.004988389017297354, P111=0.006984828709498775, P112=0.0005184264810622285, P113=0.0014849563289056594, P114=0.005562668116676625, P115=0.0008779900714638519, P116=0.0014535914773860224, P117=0.0022842502186286224, P118=0.00036421885520947537, P119=0.0020137791439441507, P120=0.0011669358689067849, P121=0.0005191103262623798, P122=0.0010299955676665597, P123=0.0003129052503620152, P124=0.0004760910475392087, P125=0.0002672717816810492, P126=0.0002812339935039489, P127=0.00013100749205639968, P128=0.0003836095421552263, P129=0.0001707953697400608, P130=0.0003058377331978003, P131=0.0002426925882107384, P132=0.0002381994037736119, P133=0.0002781305872844288, P134=0.0002339899147531166, P135=0.00034868432374522305, P136=0.00015204891304222235, P137=0.00015017389919794332, P138=7.796692308158092e-05, P139=7.330674318618682e-05, P140=6.445505333078179e-05, P141=5.505458539402352e-05, P142=6.528145321154693e-05, P143=8.07228369116419e-05, P144=0.00010434924206955323, P145=8.333002391113538e-05, P146=5.6243111573203594e-05, P147=5.658451420517674e-05, P148=4.254345123618091e-05, P149=2.5670398236130833e-05, P150=1.4121306060679437e-05, P151=1.9343854233128184e-05, P152=3.0002708202152096e-05, P153=3.3951868721729346e-05, P154=3.3152394735451456e-05, P155=1.536222279535181e-05, P156=1.3322523727787391e-05, P157=8.494520636036638e-06, P158=5.8174116888069594e-06, P159=1.2364222165808024e-05, P160=1.4063286248039478e-05, P161=1.2020547850468738e-05, P162=7.481113599439887e-06, P163=7.264071308270376e-06, P164=5.324439835108577e-06, P165=4.662687345342463e-06, P166=6.1459382751463655e-06, P167=8.020908882927283e-06, P168=8.28222046471537e-06, P169=8.211281326929388e-06, P170=5.337231027296595e-06, P171=4.409177815861818e-06, P172=4.484596507368744e-06, P173=5.022442217054692e-06, P174=8.267745085589796e-06, P175=1.03732440446271e-05, P176=1.202264455058982e-05, P177=9.684434652520118e-06, P178=5.11974090326145e-06, P179=5.634790023846623e-07),Row( P0=-566.2173099152856, P1=63.812610144336965, P2=-10.64459693482876, P3=7.577955177456857, P4=-3.7535416380006197, P5=-12.678965792844467, P6=-1.3575117870158875, P7=0.0766877034109113, P8=1.6392817192077798, P9=0.42787107397205015, P10=-4.63810042907081, P11=1.011147058266875, P12=-4.283914937013598, P13=-1.373605197551434, P14=0.27844322701637875, P15=3.721085068950975, P16=5.021583886180463, P17=2.1849640441805778, P18=5.2953785090304075, P19=4.653833247664127, P20=-1.4348351042483356, P21=0.9828972553744968, P22=0.9454731233770789, P23=2.47475072584739, P24=2.5999513028767227, P25=8.18872290913999, P26=2.0275085592911166, P27=3.3271163172310314, P28=2.074316778256396, P29=2.460683631427829, P30=3.1553152887196982, P31=3.194311651398961, P32=0.9932063650435865, P33=4.623629003036476, P34=0.7800173442233735, P35=3.231106582857215, P36=-0.1385613819343504, P37=0.9911907669744214, P38=-1.3223231969288565, P39=1.1023237281932423, P40=0.6874361381869778, P41=0.6177530409489425, P42=0.609519791833359, P43=0.6533017721347621, P44=0.6215676253146701, P45=0.6111845816883288, P46=0.5962782663087225, P47=0.5683889158995536, P48=0.6171903856154007, P49=0.6604701365881865, P50=0.7094390041322289, P51=0.7463991922335861, P52=0.1418240812373274, P53=0.007833043701965837, P54=0.0002701535215054081, P55=4.55961037273346e-05, P56=5.122671571678665e-05, P57=0.00048770027756183965, P58=0.0021589754426237224, P59=0.012220289654249388, P60=0.04724228673325038, P61=0.09214520995027198, P62=0.0989844105951657, P63=0.06085162739306757, P64=0.016533535023443978, P65=0.047998632622970085, P66=0.17641870355912792, P67=0.17277625285757317, P68=0.04774038617200174, P69=0.02505854013691078, P70=0.05547762304649329, P71=0.05473790314361964, P72=0.022771025246981384, P73=0.018311095385170454, P74=0.016755110200858722, P75=0.012394327868598554, P76=0.008863110295676763, P77=0.034561152533131044, P78=0.12719934931792265, P79=0.19413640443815233, P80=0.13399723535505412, P81=0.10575650533179907, P82=0.09419764240277581, P83=0.07452060813160566, P84=0.07969545149024417, P85=0.07077303930420421, P86=0.028761608058052914, P87=0.007374040646941259, P88=0.0073148904087694, P89=0.049607191061924676, P90=0.10473719449418266, P91=0.039870188240970564, P92=0.028897601817897835, P93=0.032439345614809684, P94=0.03959645188429368, P95=0.02040165956996382, P96=0.03686273660739047, P97=0.029752184980737797, P98=0.024110225309783816, P99=0.01182084727303379, P100=0.011306576669049983, P101=0.009338093965835204, P102=0.005756775169473287, P103=0.004880445108830613, P104=0.009745949738896571, P105=0.01853044227243905, P106=0.021996707948447703, P107=0.009544864300527638, P108=0.003966813665818219, P109=0.002827243353624516, P110=0.006034185540118067, P111=0.01761572168702755, P112=0.004309962309613402, P113=0.0013624079265136203, P114=0.012703500170885862, P115=0.015649667867266622, P116=0.002082182981189752, P117=0.003570442316046236, P118=0.014646248397002582, P119=0.005356371904790321, P120=0.005235819947963762, P121=0.01502053506382957, P122=0.01070178096717301, P123=0.0038193949786659424, P124=0.004618282498559842, P125=0.004706475460924195, P126=0.001322989816350276, P127=0.0009603557153792921, P128=0.0014335003631186776, P129=0.0016525260356302079, P130=0.0024211107323457073, P131=0.0015153803488410977, P132=0.0034626642746020102, P133=0.002857385438847895, P134=0.0035117275710419495, P135=0.003725585815652759, P136=0.002985582558601253, P137=0.0012179950347590074, P138=0.0014162624634238075, P139=0.0009351680215027995, P140=0.0009841424408694012, P141=0.0009288058619805271, P142=0.001015395972628574, P143=0.0008582753430402987, P144=0.0007393086618186505, P145=0.0010646895251554746, P146=0.0008458575868882643, P147=0.0005257428877698073, P148=0.00027119481954799023, P149=0.00014251417386213557, P150=0.000156855206229947, P151=0.00013905640888513886, P152=0.0001337108819637483, P153=0.00023517825042917667, P154=0.0003698765430220206, P155=0.00036833970109632837, P156=0.00028651359626684933, P157=0.00029158981464183053, P158=0.00023651983835000498, P159=0.00019255209222206196, P160=0.00017458355415884288, P161=0.0003051186211307556, P162=0.00035899090244515355, P163=0.0003677008793758103, P164=0.00013806315005588594, P165=0.00021286467105560415, P166=0.00014882161643127665, P167=0.00011196686219296686, P168=0.0001558218073598372, P169=0.00012286930722973168, P170=5.192640286141822e-05, P171=4.488277428703188e-05, P172=4.9429939009303335e-05, P173=6.806590071531727e-05, P174=7.559196154171486e-05, P175=7.549985721960555e-05, P176=6.958745827344457e-05, P177=5.9614315247860804e-05, P178=3.295138131844941e-05, P179=4.842607908069539e-06),Row( P0=-515.919918736642, P1=38.43712443455467, P2=-15.208510957403304, P3=6.800136444709296, P4=-5.165073079818386, P5=-11.201333845991481, P6=1.473345682011263, P7=-0.49437727498993933, P8=2.127883906567025, P9=-0.43022968071415946, P10=-6.4481530545598655, P11=2.176258961493529, P12=-4.369762064711962, P13=-0.6747272139559322, P14=-2.314967318821007, P15=-3.412223423521941, P16=0.4459368042801847, P17=0.9637986313651016, P18=6.993712457710158, P19=13.428637417670291, P20=11.547944397172849, P21=14.06178104169231, P22=10.91124141267557, P23=5.6513144144848795, P24=-0.3242753056045868, P25=1.846576202199375, P26=-0.8757692044026782, P27=3.5677352270971756, P28=5.056540411947054, P29=4.00600931071414, P30=-1.657920448154641, P31=-0.7667009659616116, P32=-4.002912040682473, P33=1.382066885142923, P34=-1.2097451959178311, P35=3.332169182342808, P36=-1.6442448042607416, P37=-0.4233052754994559, P38=0.06831806379085154, P39=1.500945957124918, P40=0.599549554481971, P41=0.569581447696967, P42=0.6600441288900545, P43=0.7053005598441138, P44=0.662044766209081, P45=0.6576598040900663, P46=0.7072667524018003, P47=0.7687362072014787, P48=0.7554475801713268, P49=0.7372703795170851, P50=0.6603546429532989, P51=0.6471578016532842, P52=0.018703468208589383, P53=0.0033102024049211282, P54=0.0005936691617837566, P55=0.00020394531324152742, P56=0.00026239042682094245, P57=0.0003095397807197935, P58=0.00026697579613901027, P59=0.0018095677300980124, P60=0.013707983603718799, P61=0.059958559641839396, P62=0.21578754825677518, P63=0.6362917272994172, P64=0.6172422001270723, P65=0.4707418990926358, P66=0.16847963764581608, P67=0.022303210966241934, P68=0.004950377121030845, P69=0.0027723505991796497, P70=0.003666404992901124, P71=0.003028339725077789, P72=0.00589395086573517, P73=0.012114269228641708, P74=0.046365677704263246, P75=0.18556750917237255, P76=0.5257928977011335, P77=1.9076517911604085, P78=1.6169703567121934, P79=1.450454371189329, P80=0.8979967844840441, P81=0.22078541220917852, P82=0.036755331898587255, P83=0.005257321947140715, P84=0.005786422237539714, P85=0.020439377801242158, P86=0.16675126090070544, P87=0.790128251217409, P88=1.3989897543709564, P89=1.860152816753793, P90=1.1519629480151294, P91=0.19498026805997132, P92=0.05655769814460395, P93=0.008075248763539454, P94=0.00995363203934856, P95=0.2833921900055891, P96=0.7282540372862465, P97=0.21898573431541132, P98=0.19393759622525517, P99=0.05478235370154729, P100=0.005627084681486752, P101=0.0033592104620280656, P102=0.014207314614519417, P103=0.06159254715000217, P104=0.11148561674354195, P105=0.10595134254515669, P106=0.058230196251499444, P107=0.008029147288395364, P108=0.128165106810095, P109=0.15380159120271994, P110=0.09164519718564446, P111=0.036303564514036206, P112=0.0189295416489789, P113=0.08877218145534219, P114=0.10824493206303587, P115=0.05713002429257027, P116=0.01985868389432845, P117=0.09644286022016983, P118=0.0879705540484893, P119=0.0351165382118648, P120=0.020286598333168114, P121=0.05574364301246401, P122=0.059885087669704504, P123=0.02020063287270342, P124=0.02906667589884167, P125=0.06554396032651635, P126=0.017370865716098596, P127=0.011683562699697775, P128=0.00629522980366559, P129=0.010706658630096087, P130=0.023338712797557754, P131=0.017289224350577975, P132=0.04283416389514869, P133=0.027887967390252986, P134=0.04990398904136274, P135=0.05078507701360931, P136=0.04979516978131529, P137=0.023862499832639165, P138=0.00766755333335434, P139=0.0069855439400387034, P140=0.007374378662973035, P141=0.009360708406376317, P142=0.011596793664681544, P143=0.010493013995532466, P144=0.008662283214327875, P145=0.004774229183669811, P146=0.006732222998160885, P147=0.0022211350068358717, P148=0.0015877915190825654, P149=0.001153341020236021, P150=0.0006960189632880735, P151=0.0006486071781550252, P152=0.000848113542055546, P153=0.0010857054582520933, P154=0.0009092420197903926, P155=0.0007856477660933091, P156=0.00044932959757461956, P157=0.00045537751804555874, P158=0.0004567466824543292, P159=0.000505186193775914, P160=0.000849833044268286, P161=0.0005528137948673677, P162=0.0003384984732002139, P163=0.00024481204484608714, P164=0.00016995894877958428, P165=0.0001953587045695006, P166=0.00034042629356628674, P167=0.0005028870900171173, P168=0.00035320747415591117, P169=0.00026747590565096723, P170=0.00030390066314605365, P171=0.0002546795793141657, P172=0.00021765085635981052, P173=0.0003255035325564373, P174=0.0005946740250364247, P175=0.0005045539190457739, P176=0.0005933285749915454, P177=0.0008134000176912091, P178=0.0004954989310032371, P179=3.698199436252404e-05),Row( P0=-663.478977747077, P1=75.46551651354086, P2=-3.542661853971601, P3=8.87002069263449, P4=-2.018127664283605, P5=-2.325304628579026, P6=3.24369763921376, P7=2.3513712750212314, P8=2.13537197150189, P9=3.576074570041566, P10=1.8065760980721384, P11=-0.7934110616732305, P12=0.9159454179547264, P13=0.1687113109521656, P14=-4.859026500299955, P15=2.0283461128579887, P16=-0.5056901151538538, P17=1.4944858184651029, P18=2.3063084554007744, P19=1.7254128981863321, P20=-0.47602938223106706, P21=-1.5283689549916348, P22=0.9257979367445748, P23=-0.21095639372067515, P24=-0.2804041095780229, P25=3.4583957813694677, P26=0.4468306501957153, P27=5.406450285536998, P28=-0.42889963732099134, P29=3.571408206526998, P30=1.0184780883936764, P31=4.148404924708938, P32=2.371480956316068, P33=7.728137774048766, P34=5.299320337626189, P35=5.8863404353220075, P36=2.0440473502659975, P37=0.5172992703747961, P38=0.2480365802274149, P39=1.2807089753492742, P40=0.7145789201802909, P41=0.6527802371346861, P42=0.6565673476685566, P43=0.7179022763831163, P44=0.7118433305794885, P45=0.6867470739948037, P46=0.6892522682587544, P47=0.6759168069543181, P48=0.6837689022422622, P49=0.6902240291684205, P50=0.7426404146485964, P51=0.768585331392144, P52=0.0013801778775960059, P53=0.00017345721059170552, P54=2.8415481634843144e-05, P55=2.4794995401309115e-05, P56=5.8837044894099356e-05, P57=0.0006663784537735103, P58=0.006385073481032734, P59=0.017692079492772475, P60=0.025283341505658363, P61=0.04353627746263339, P62=0.027152928366112185, P63=0.004543326711844353, P64=0.001072674006668172, P65=0.0038045237514456325, P66=0.01511135336467035, P67=0.023784682949903293, P68=0.01023308108586839, P69=0.005247491388150081, P70=0.0070722401037898236, P71=0.009101542242938439, P72=0.010256557450148028, P73=0.006938354342604077, P74=0.004831438295962813, P75=0.005922550230740581, P76=0.0037958649198720144, P77=0.0029268630549792118, P78=0.00366415599265018, P79=0.005560151343872687, P80=0.010574045949924138, P81=0.019787753407147415, P82=0.03844344361938593, P83=0.0437247769396874, P84=0.03510066204034214, P85=0.011282228529584335, P86=0.0011684205969932874, P87=0.0016896251984731892, P88=0.006716802848156694, P89=0.01631246677060443, P90=0.010363009395281416, P91=0.0026242322589024735, P92=0.00188399822202991, P93=0.0007965765734967013, P94=0.006472231580296285, P95=0.01245571704211817, P96=0.019758279288609806, P97=0.007976218781724243, P98=0.0012459252267498669, P99=0.001168017805895372, P100=0.0015172023846189275, P101=0.004695744070276743, P102=0.005997873960362108, P103=0.0021087099232901647, P104=0.001129646342026673, P105=0.0010848808858601481, P106=0.012674786865085552, P107=0.006510511278645487, P108=0.0012372520546849435, P109=0.00039538130909527036, P110=0.005202498309951542, P111=0.0058567661986052505, P112=0.0006296719351664314, P113=0.00037775881755096915, P114=0.0009560130256076419, P115=0.0005478725480962315, P116=0.0007316109561414314, P117=0.0004591128038476723, P118=0.00022021245960491862, P119=0.0003864518073489951, P120=0.00045780902807098, P121=0.00041423301042182184, P122=0.00015070726775941743, P123=0.0001893410102893123, P124=0.00012344170504217654, P125=7.121397934768576e-05, P126=4.2912524388650766e-05, P127=7.341309006620749e-05, P128=5.2024808934452526e-05, P129=5.9548365026795506e-05, P130=0.00010272885482406526, P131=0.00012304096585986956, P132=0.00012866167044014846, P133=0.00021978892923946413, P134=0.00018873902379260598, P135=0.00030565345219910013, P136=0.0001997148188898027, P137=0.00017665103988206504, P138=7.256737812638697e-05, P139=5.881123638490072e-05, P140=4.3604954758311e-05, P141=4.606988907370052e-05, P142=3.187418654645364e-05, P143=3.8691899939376505e-05, P144=4.507401302674314e-05, P145=4.3955076421688295e-05, P146=3.011966092498503e-05, P147=2.209232210900346e-05, P148=2.1009585828604948e-05, P149=1.5812266004456103e-05, P150=1.0169009507668416e-05, P151=1.453626489802439e-05, P152=2.0158736903048994e-05, P153=3.16260170819901e-05, P154=3.672050065592401e-05, P155=4.347185053279081e-05, P156=7.172464363809037e-05, P157=0.00010994735062621423, P158=6.724849152017764e-05, P159=6.067114671617487e-05, P160=2.7192771563881077e-05, P161=4.980904716034398e-05, P162=2.6899842348217285e-05, P163=1.6909195006543306e-05, P164=1.4341829667673051e-05, P165=1.5120021126190787e-05, P166=1.8637737329621137e-05, P167=1.702085470565441e-05, P168=5.148734663831543e-06, P169=5.973510284063214e-06, P170=3.901399820442853e-06, P171=4.134549787450723e-06, P172=2.2610744208832023e-06, P173=5.1248943772112476e-06, P174=6.164648337795286e-06, P175=5.48819719694048e-06, P176=6.263874110998236e-06, P177=6.1262428883580235e-06, P178=3.252719321828747e-06, P179=2.7313180447626184e-07),Row( P0=-591.0346118294734, P1=55.082189057726396, P2=-16.867939838069034, P3=16.343985338987583, P4=-5.708876411053164, P5=-8.31108697117522, P6=5.65540124067595, P7=2.058903427574913, P8=2.0038649356800056, P9=2.489209317643224, P10=-5.0197106494260595, P11=1.3376565607125965, P12=0.5339067012932018, P13=-0.1584390802949611, P14=-1.6892302643072208, P15=1.8146735554787397, P16=0.37361527332408967, P17=-0.9078788783630893, P18=1.8678517589210593, P19=1.266708396627607, P20=3.039768733954509, P21=0.08895114966271758, P22=3.55641121339365, P23=2.4751581266603675, P24=0.2392158925441297, P25=4.021097785208694, P26=-0.04103720849586299, P27=3.125237903125055, P28=3.1815738975477945, P29=7.23307090848833, P30=3.2381067704480144, P31=5.0215428764255, P32=1.058220087322525, P33=1.7086090480461262, P34=-2.023932361059519, P35=2.527003281066905, P36=0.7590197990570074, P37=2.4109797093234593, P38=1.3774000651797853, P39=0.19171902627301482, P40=0.6486121411990894, P41=0.7097754100184315, P42=0.7228873586713233, P43=0.746628668007305, P44=0.7405976432874554, P45=0.745045491359419, P46=0.7186721586787019, P47=0.6766179446683804, P48=0.6641957277606377, P49=0.7029775244493638, P50=0.6850352456064913, P51=0.6694830685423346, P52=0.012263658420774056, P53=0.0028947967884581263, P54=0.00018435479797184956, P55=6.1459777937202e-05, P56=0.00025830350640746206, P57=0.0017154881067355248, P58=0.006837805071868325, P59=0.020273480360578734, P60=0.0628784392370748, P61=0.07573665386540294, P62=0.19283953456879316, P63=0.25669948522787556, P64=0.07842317020566426, P65=0.016980550782199507, P66=0.015148034860126445, P67=0.01785477043698041, P68=0.03350861053766241, P69=0.07800832313331613, P70=0.0813228589706381, P71=0.03945066917363037, P72=0.045650309692726695, P73=0.0620615690922625, P74=0.1008581967427917, P75=0.1169213068874463, P76=0.11323505065487674, P77=0.12119473877524531, P78=0.09920460786537114, P79=0.1109339963014454, P80=0.05452155412075491, P81=0.020701785866829362, P82=0.01611110146537779, P83=0.028794770448652158, P84=0.05295131816250709, P85=0.1045197742668764, P86=0.13663559331536493, P87=0.1846991599440152, P88=0.16330153473800416, P89=0.09341659804229144, P90=0.0660857236845574, P91=0.06429265181661734, P92=0.23654035833052614, P93=0.16390841725408106, P94=0.15289153329477292, P95=0.13128224307655362, P96=0.0549306384976859, P97=0.012819481088408084, P98=0.020145327807303038, P99=0.059373866906186505, P100=0.017897046122854227, P101=0.006712837261835316, P102=0.006930563994328349, P103=0.015710247017894606, P104=0.006094061369645636, P105=0.016557917579610348, P106=0.013750025858521274, P107=0.005684027935325004, P108=0.007250101276525113, P109=0.03184432200665563, P110=0.03780108251307946, P111=0.01880562518559288, P112=0.005721371183863348, P113=0.02806968542137848, P114=0.061807509331979815, P115=0.020242978469467193, P116=0.025690291233632187, P117=0.03366242476637214, P118=0.008544162248613622, P119=0.02003940443104404, P120=0.012164731189215142, P121=0.014293914922369255, P122=0.010823869036035294, P123=0.006193651129150189, P124=0.01618535377734046, P125=0.011934634077479066, P126=0.006715685511167506, P127=0.00635472037921549, P128=0.006217949163175459, P129=0.003064915475605494, P130=0.006623233418717721, P131=0.013062922761435964, P132=0.01797875872883259, P133=0.02332586931128707, P134=0.017327300906264574, P135=0.01843936694222649, P136=0.0045157055757160175, P137=0.005838499630455485, P138=0.003974620352692844, P139=0.006504619044825518, P140=0.00878785612122535, P141=0.01134544518727766, P142=0.006500921496349439, P143=0.002433869057217315, P144=0.0014377365711815038, P145=0.0021059338805478954, P146=0.0013217609766652543, P147=0.0007163566912684485, P148=0.00041658051296234377, P149=0.0005438136453024682, P150=0.0008344110463994798, P151=0.00041060707825057795, P152=0.000532422050869079, P153=0.0007236299597889084, P154=0.0007954727998557427, P155=0.00042718260090638554, P156=0.0005425242691623095, P157=0.0007827827432272413, P158=0.0009438916555213607, P159=0.0007571299157560764, P160=0.0006666554539707243, P161=0.0005865466589164996, P162=0.00045356653237207603, P163=0.00030314475148783733, P164=0.0004689247087042345, P165=0.0003235041360493671, P166=0.000384289290350752, P167=0.0004981352385128356, P168=0.0007555048150439815, P169=0.0008676726945195433, P170=0.000375763926266278, P171=0.00019912496763442478, P172=0.00020874166943294704, P173=0.00014180878487033666, P174=0.00018784669079041457, P175=0.00029045724144181835, P176=0.0004504243291061613, P177=0.00041984640880540666, P178=0.00020261291137261685, P179=2.813464441297912e-05),Row( P0=-593.1931111730134, P1=60.41501971889513, P2=-1.7594824937990576, P3=9.880223482485203, P4=-0.2089702469592872, P5=-5.482283534234039, P6=5.3599871282846365, P7=8.67085509181714, P8=6.6222336431368785, P9=-2.127046132319929, P10=1.4142036086788545, P11=5.310527180420676, P12=-0.8652605061346293, P13=2.4410761095572937, P14=1.052911773052525, P15=0.17653743379026174, P16=1.8413490174521547, P17=4.443309789949671, P18=5.181330909525488, P19=5.172793217061208, P20=4.099290853567109, P21=5.841704192843846, P22=4.320034419438243, P23=3.719138393969302, P24=2.0728636153284765, P25=6.0683131315318635, P26=2.901523117276152, P27=6.284080541430637, P28=4.424152709450804, P29=5.337236892961898, P30=2.9419709853327802, P31=3.7004715745266465, P32=1.4765372881843015, P33=3.573797662316219, P34=-0.339893195928226, P35=3.4945130137844482, P36=0.9279444526845536, P37=1.3101202737079767, P38=-1.6936946916569817, P39=0.848756190725535, P40=0.677282144685846, P41=0.66839775851537, P42=0.6916118494921233, P43=0.6825028572574703, P44=0.672027362061045, P45=0.7170092720755916, P46=0.7249317431871423, P47=0.7005829307438839, P48=0.6941578568327238, P49=0.7333696376231108, P50=0.7729875744550216, P51=0.7350306964396932, P52=0.2279268796614648, P53=0.012479893095297682, P54=0.0006625657324532606, P55=0.00012946936613648122, P56=0.00012150483218931875, P57=0.00027131738217794405, P58=0.0015448767918212973, P59=0.011790886517219741, P60=0.04420318288889398, P61=0.07318582248198, P62=0.10183584585816487, P63=0.1435221541595044, P64=0.0703449721947087, P65=0.06603592402306192, P66=0.050203569001955106, P67=0.01933524413921884, P68=0.011583833260112024, P69=0.026684870163048227, P70=0.04175985124141183, P71=0.02414596088019835, P72=0.022369368841625673, P73=0.023779805612030525, P74=0.022290706842166424, P75=0.031122196560109103, P76=0.02517127198627168, P77=0.02915958633374229, P78=0.06101254661298522, P79=0.11455196911112502, P80=0.10251881375101686, P81=0.05206831735762839, P82=0.03442383786106265, P83=0.030470133806154678, P84=0.027246933263816834, P85=0.02027474221782668, P86=0.02069141726886997, P87=0.05930303136425016, P88=0.08346062950889135, P89=0.025044587317224757, P90=0.010683785051545129, P91=0.015635185345129456, P92=0.03506658126703828, P93=0.03261879397964316, P94=0.04574638306548513, P95=0.03256460637950178, P96=0.048718634660232255, P97=0.04207126499921613, P98=0.010044925952820954, P99=0.0072957577223519325, P100=0.004533567271136187, P101=0.003672162143086168, P102=0.0022045718227159217, P103=0.006078990419033352, P104=0.004089659957160412, P105=0.0019934070857779746, P106=0.001819146466280756, P107=0.0019802039784582604, P108=0.005992957096490252, P109=0.017609134609556328, P110=0.014166628435455879, P111=0.011810180106514339, P112=0.002659819467446486, P113=0.003141648428172751, P114=0.0071052487096725046, P115=0.0038482810195525482, P116=0.002869673204267666, P117=0.0022118520161422163, P118=0.0023007256450580183, P119=0.001292487722358775, P120=0.002408175539144328, P121=0.0032825029948634864, P122=0.002541360249376755, P123=0.0018094215414563583, P124=0.0015346216707512377, P125=0.0013806971558234487, P126=0.000831961923720284, P127=0.00144166457699575, P128=0.0014785342354908864, P129=0.001793206081231105, P130=0.0013666840285806309, P131=0.0006244620406755725, P132=0.0008530705882883951, P133=0.0009717971793989772, P134=0.0005053867394312278, P135=0.0007343050409292979, P136=0.0003016128875330822, P137=0.0003631954064053514, P138=0.00020352374580434598, P139=0.00022729107024953182, P140=0.0002479776719232244, P141=0.0004011370274301749, P142=0.0006620893008170152, P143=0.0009612428619392341, P144=0.0007427626702583535, P145=0.0004509193647867584, P146=0.0003935176570107239, P147=0.00035242825142053696, P148=0.000356939279355009, P149=0.00020586808097985788, P150=0.00012503287880348237, P151=0.00014139111417060972, P152=0.0002416047036136648, P153=0.0004467620914420485, P154=0.000559540779459437, P155=0.00036779785869573073, P156=0.00029696664680499126, P157=0.000279973979656347, P158=0.00018719062576534387, P159=0.00014635861331307605, P160=0.0001504465583876112, P161=0.00014673146394471076, P162=0.00013467960563700579, P163=0.00011653784658810681, P164=5.998613904040555e-05, P165=3.5624526356247586e-05, P166=3.8429756572402506e-05, P167=5.1669742265829235e-05, P168=4.822740304980568e-05, P169=5.109907401954925e-05, P170=5.4919050181122936e-05, P171=4.4210018767875084e-05, P172=4.1570588195494864e-05, P173=5.83034861643256e-05, P174=9.118736389037643e-05, P175=9.1193444728342e-05, P176=5.496627395764938e-05, P177=7.985563642016174e-05, P178=4.796724430317905e-05, P179=5.533458429578381e-06),Row( P0=-616.7486742540591, P1=73.47953290849716, P2=-26.32194460997601, P3=13.238511133406313, P4=-3.4322857385434693, P5=-8.77693085683926, P6=2.761985310834042, P7=3.3400642640410236, P8=4.753544110523251, P9=-0.6598801352603457, P10=-2.908904705618149, P11=2.8244196732137037, P12=-1.177614191642372, P13=-1.0701981469769184, P14=1.1738792088887737, P15=4.912670983357212, P16=2.0681434584082674, P17=-1.505593113500684, P18=2.2910783521552625, P19=5.717791398207093, P20=2.4793946123355735, P21=5.0283063067839615, P22=2.5153720629314575, P23=-0.8834015226220292, P24=1.8962325605212533, P25=-1.0128929778910085, P26=-1.7687558329675714, P27=1.5898429345021574, P28=1.4285606895044898, P29=3.663627085483392, P30=1.0922711274980745, P31=2.190254147822732, P32=-1.0673588776994527, P33=3.0094536050141194, P34=0.22972433703259082, P35=4.984512679110857, P36=2.3082456370551605, P37=2.2252203939760107, P38=0.3580831350359762, P39=1.0462183464299888, P40=0.69036668990668, P41=0.6366331602247166, P42=0.6466644745430256, P43=0.60420289867989, P44=0.6102357425032295, P45=0.6775852316294778, P46=0.7186451267292585, P47=0.6569994820008095, P48=0.6718262487680489, P49=0.6995689978937484, P50=0.7241886853902848, P51=0.7520952540749194, P52=0.0002947729559099674, P53=0.00015259416629506125, P54=8.348935030463666e-05, P55=3.9225192417713115e-05, P56=4.172973512979811e-05, P57=0.000602254386253986, P58=0.004819361453589785, P59=0.010272384277391633, P60=0.01369043361381943, P61=0.011976402370484696, P62=0.015486732260404576, P63=0.023810837877030866, P64=0.012392143253168278, P65=0.011686298006685282, P66=0.028902789842039932, P67=0.03279748417931264, P68=0.014638265045570859, P69=0.006516303280291921, P70=0.009003483080432093, P71=0.0033569980894408193, P72=0.0018860865646457394, P73=0.003990474101197098, P74=0.006275417637846167, P75=0.010288361279016443, P76=0.011415653455512086, P77=0.010410098425082353, P78=0.008090023266330596, P79=0.00942727198551464, P80=0.007490063631199297, P81=0.007485876426028859, P82=0.016104170846483117, P83=0.03994806844597497, P84=0.03564122052876583, P85=0.025520066310035703, P86=0.017212986298966157, P87=0.017342578197879002, P88=0.036666881862100084, P89=0.014228387141173573, P90=0.004718395359642041, P91=0.003195455467442925, P92=0.004322258772617294, P93=0.012734377477804664, P94=0.02633649797222299, P95=0.04377262473173446, P96=0.008021328992451006, P97=0.002151286044855783, P98=0.00615272045559042, P99=0.0033187678920946094, P100=0.012529342689178031, P101=0.007651725480453539, P102=0.0015539826314027044, P103=0.0022853845800787256, P104=0.007835753305150467, P105=0.029172566062024945, P106=0.014375561762161867, P107=0.0012461742218913773, P108=0.004746211818813763, P109=0.009212800281866506, P110=0.003450290786150032, P111=0.002559298265559982, P112=0.006499021718947176, P113=0.0020968654756433635, P114=0.00393527636405455, P115=0.009053518849998957, P116=0.003312767365812972, P117=0.005144809123929238, P118=0.003996342346710284, P119=0.0021935335014493336, P120=0.006112036190894678, P121=0.001961426472599927, P122=0.0021213946147473177, P123=0.0010322874335335693, P124=0.0007004389788136253, P125=0.0006278270703398856, P126=0.0002568100203746234, P127=0.0002328390674459061, P128=0.00035593792374750776, P129=0.0003310755701188767, P130=0.0003925885909385676, P131=0.00037360433695955345, P132=0.0005130932285759968, P133=0.0007074769798011142, P134=0.0013121615879832934, P135=0.0016291195751393013, P136=0.001058324733756415, P137=0.0006932202970080718, P138=0.0003081933960214667, P139=0.00022403973681887088, P140=0.00014356793765976913, P141=7.939960489897522e-05, P142=7.22634021106201e-05, P143=6.732486406205244e-05, P144=6.808980491081221e-05, P145=8.627644332049879e-05, P146=0.00010462830354241439, P147=6.230941513808122e-05, P148=2.458690220462487e-05, P149=1.6090731111954015e-05, P150=1.2469395919801369e-05, P151=1.327034804456754e-05, P152=1.7705003129607048e-05, P153=2.6787596869336134e-05, P154=2.3257709322346303e-05, P155=3.35769807306006e-05, P156=2.579274201761718e-05, P157=5.408812420770593e-05, P158=5.369874993999142e-05, P159=3.728383558039675e-05, P160=3.0263349759827916e-05, P161=3.519532347941036e-05, P162=2.9885499674563566e-05, P163=1.9938585031682337e-05, P164=1.4137213486345818e-05, P165=1.8837137499343653e-05, P166=1.1549917251168622e-05, P167=8.264997428672802e-06, P168=6.987510747400949e-06, P169=5.8711792548080696e-06, P170=3.983850933547123e-06, P171=4.572057901832278e-06, P172=3.96729256225459e-06, P173=3.872202793410536e-06, P174=3.7068147638040195e-06, P175=5.7653201203866805e-06, P176=5.503942613652804e-06, P177=5.147033547345294e-06, P178=2.639034963988468e-06, P179=3.6578494369330935e-07),Row( P0=-625.5059332229666, P1=70.64197939146919, P2=-11.344893494342495, P3=14.177200142613188, P4=-1.699203164367986, P5=-5.440688874838289, P6=5.396242048716508, P7=0.3110149518460817, P8=3.5045847413304583, P9=3.8506414715843724, P10=-2.5249303167512602, P11=2.055036714382025, P12=0.8227317870285099, P13=1.9799950114725027, P14=-1.1760794988414953, P15=0.7422315059922912, P16=1.078511558746926, P17=0.0228968176036516, P18=1.3082997623326595, P19=1.407295767966735, P20=1.5793320135560502, P21=-3.2878093044763874, P22=1.9992052591749885, P23=0.2801401240155723, P24=-1.1526100635760217, P25=5.876686787658117, P26=4.926294383422165, P27=8.353911721855702, P28=5.530844648664527, P29=6.286077085176018, P30=1.4126290753948243, P31=3.1367088307234092, P32=-0.49971880670668356, P33=2.7246532918255992, P34=1.3366163920659189, P35=2.948062406333787, P36=-0.589448170168127, P37=0.6433387308635167, P38=-0.5725482104025176, P39=0.6458449130679013, P40=0.6576204932527392, P41=0.6730506542793785, P42=0.7267665910405898, P43=0.8075449173260957, P44=0.8060770605241245, P45=0.7312757982865798, P46=0.6982676627907396, P47=0.704577003220265, P48=0.7067107362055708, P49=0.7314773348119288, P50=0.6998530545483317, P51=0.69286243195489, P52=0.006937739840266246, P53=0.0005661085455179716, P54=9.416456694492108e-05, P55=3.084709603085929e-05, P56=7.611617027707719e-05, P57=0.0008612219356329957, P58=0.00329227077331226, P59=0.013538743136967818, P60=0.06666436318512176, P61=0.11215168586548328, P62=0.06620005163795499, P63=0.014583330078792943, P64=0.0029145771777820562, P65=0.0031793131298484597, P66=0.0058318623406267735, P67=0.008662059418758253, P68=0.01746871592613371, P69=0.04689628466494336, P70=0.059214841037507265, P71=0.05257727899831767, P72=0.032471712741198575, P73=0.014435261711499201, P74=0.005940579859483547, P75=0.003962377278155059, P76=0.004530098099881214, P77=0.020039532830228555, P78=0.05436036722519685, P79=0.060598994719943754, P80=0.0398577431545746, P81=0.03809634758399353, P82=0.042908146839926636, P83=0.0212839006799477, P84=0.020232053623128854, P85=0.02353524981278746, P86=0.0144049220414967, P87=0.03136233706172841, P88=0.05541604912631811, P89=0.09152939659623477, P90=0.08855899939114208, P91=0.03800008075436655, P92=0.016935125776270963, P93=0.012997228166175087, P94=0.013571759768575073, P95=0.09513041531920813, P96=0.15245369751979806, P97=0.1347115792482471, P98=0.017781074419880683, P99=0.002409234902466977, P100=0.005715735462005662, P101=0.012752850443792834, P102=0.011773701944580698, P103=0.007589633722100048, P104=0.0018884592048663744, P105=0.005984204545343427, P106=0.013446094086958757, P107=0.019269947826737487, P108=0.013864810343245096, P109=0.00502796565823154, P110=0.021216336353945195, P111=0.04520432654834466, P112=0.01191503719526328, P113=0.012532266538359994, P114=0.027817715540264933, P115=0.007213769339883673, P116=0.007001884207334543, P117=0.028142031503276654, P118=0.003435093795991197, P119=0.004001261939965743, P120=0.019533407598450285, P121=0.009929303240890886, P122=0.0011877361015811946, P123=0.001305558713275856, P124=0.0007717485445287074, P125=0.0008714745652844245, P126=0.0005540151182599893, P127=0.0010123154373106132, P128=0.0006177132151756611, P129=0.001079749246883996, P130=0.0014440424766261374, P131=0.0046332194984228334, P132=0.0029545893985952463, P133=0.007047985846598548, P134=0.006771633955670155, P135=0.0020001912429049953, P136=0.001171186800795622, P137=0.0006260995792413984, P138=0.000648111300300633, P139=0.0008072249927714909, P140=0.0007890816795918183, P141=0.0012774491655494742, P142=0.001420792181618279, P143=0.0009596013263417978, P144=0.0006753213925394744, P145=0.0005745206343034555, P146=0.0005541820676442532, P147=0.00015157563200404213, P148=4.736620220053648e-05, P149=2.0204867082300218e-05, P150=1.1793259307868323e-05, P151=9.097401761872529e-06, P152=1.6718431088090553e-05, P153=1.210534327561215e-05, P154=1.4291850323973232e-05, P155=1.6328915452670114e-05, P156=8.80639267244222e-06, P157=9.065132866172151e-06, P158=1.509891051751313e-05, P159=6.87649666097148e-06, P160=6.074737586840302e-06, P161=6.621667378239565e-06, P162=7.601627991637217e-06, P163=1.1053822317539703e-05, P164=9.132640956824047e-06, P165=2.043573485082595e-05, P166=1.1455229134526446e-05, P167=1.3950064439376788e-05, P168=1.0394695265282893e-05, P169=4.736894186115441e-06, P170=3.468742352267901e-06, P171=3.222339843533675e-06, P172=2.3669079166271517e-06, P173=2.395873337959296e-06, P174=3.226321083105e-06, P175=6.234498594963407e-06, P176=7.011119634840001e-06, P177=5.077775921363078e-06, P178=1.974628047966851e-06, P179=2.426335402610162e-07),Row( P0=-632.0316209475742, P1=68.75039098454964, P2=-16.253515938370075, P3=13.078709577892614, P4=-4.68630417914809, P5=-9.305168104125473, P6=4.543541141667138, P7=0.47383776823259316, P8=1.2070180657565044, P9=2.9649310577536183, P10=-4.49788631133165, P11=0.9636833002839031, P12=0.8421184830798412, P13=0.06643608012573296, P14=-0.07466196607630196, P15=0.8484872548095603, P16=-0.6111297126670429, P17=0.4981350466616517, P18=0.2716217172323063, P19=0.3084876882819722, P20=0.3766328653465535, P21=-2.138151435499868, P22=2.469190858393939, P23=0.7979893508231231, P24=-0.7792787969677496, P25=2.1022756697165557, P26=0.2542470587859048, P27=2.761593198278103, P28=3.343090465561398, P29=5.745862790756492, P30=3.7794725505687503, P31=5.483663754336833, P32=2.550284979887867, P33=2.789190350588906, P34=-0.14774806402267798, P35=2.8394866966341032, P36=0.8108648412068414, P37=2.2471671542567764, P38=1.6398822952036767, P39=2.3320289660960527, P40=0.7613629109271706, P41=0.7517702444997657, P42=0.7640621980846409, P43=0.7306256828795574, P44=0.7535397964874884, P45=0.7345368050759413, P46=0.7528101114387201, P47=0.7240396556700359, P48=0.7430691695639653, P49=0.728561051150592, P50=0.7495292207794426, P51=0.7231276234673888, P52=0.0006808632812334147, P53=0.00027263209263144497, P54=6.70094840677235e-05, P55=2.5646734281156208e-05, P56=7.829022669091534e-05, P57=0.000762225639045322, P58=0.0037945113500765315, P59=0.015630375337291863, P60=0.048526082065430305, P61=0.06555695334353574, P62=0.040322438677958185, P63=0.016986607352253297, P64=0.003917768324843771, P65=0.004480697071905297, P66=0.008798033313374474, P67=0.01212943942992324, P68=0.02647331467860158, P69=0.04331066897324802, P70=0.05537806558539094, P71=0.03931050311910698, P72=0.0148098436372992, P73=0.009629338980494983, P74=0.014404472541926363, P75=0.01675752839594568, P76=0.01610345466252232, P77=0.032504228104956755, P78=0.021719785461593994, P79=0.01924772257984243, P80=0.0386575657939357, P81=0.05278982569018444, P82=0.03985458182791004, P83=0.016156687440740215, P84=0.035776904473662795, P85=0.06821916205929485, P86=0.03197244970476699, P87=0.010073526776966182, P88=0.02696533583009053, P89=0.01870847563236736, P90=0.010702930904435916, P91=0.02875263696223395, P92=0.0783698340641885, P93=0.04730897011869452, P94=0.03082686876128587, P95=0.10405612193214107, P96=0.04310185754965968, P97=0.007753674414551734, P98=0.03491133450449083, P99=0.013399999500300495, P100=0.008376442740164261, P101=0.014479608269462619, P102=0.0032195152537323637, P103=0.006512778713172455, P104=0.001440798597328837, P105=0.006957739551378386, P106=0.010916565325120312, P107=0.0036517832075823706, P108=0.0033085034286188474, P109=0.013287699236864011, P110=0.017371286657715167, P111=0.009101108978642282, P112=0.005030869183263421, P113=0.003438861375808343, P114=0.009682465004241884, P115=0.0059268077938274604, P116=0.0031761979677087074, P117=0.007404769582859387, P118=0.001556622297658135, P119=0.009576201215269569, P120=0.007201798608273492, P121=0.0035061960141645246, P122=0.005286094560799493, P123=0.0012648277419074535, P124=0.0024469029490235697, P125=0.0006025249123859395, P126=0.0010387638935821385, P127=0.0016468691273762743, P128=0.0018943088542668159, P129=0.001260338046043807, P130=0.0020001536861644774, P131=0.0028380074615894746, P132=0.0031167486815125247, P133=0.0031464524579822477, P134=0.003450037755280258, P135=0.0019680036944434917, P136=0.002816202485781376, P137=0.0013339705880071474, P138=0.001228867445031997, P139=0.0018147587497671835, P140=0.0018396521620945075, P141=0.002498991463661662, P142=0.001322263089445298, P143=0.0006596121214689898, P144=0.0004409273038136287, P145=0.00047991757277974367, P146=0.0004180410718023891, P147=0.0001345857744835458, P148=3.6393835482172746e-05, P149=1.2273698866273564e-05, P150=7.174403841097604e-06, P151=1.877897938599311e-05, P152=2.4753398565072688e-05, P153=1.976913713636249e-05, P154=2.6706196110225347e-05, P155=1.5455083099883068e-05, P156=1.193972772585349e-05, P157=1.588698442943258e-05, P158=1.5423887443845214e-05, P159=9.199474212483647e-06, P160=1.427571169029386e-05, P161=1.6776072188615962e-05, P162=1.1335346054352223e-05, P163=6.8709001046211175e-06, P164=5.859292600181662e-06, P165=6.6545552858612115e-06, P166=6.526360655977553e-06, P167=7.316384941359195e-06, P168=6.0715377724877025e-06, P169=4.327069481227143e-06, P170=3.0757441005415983e-06, P171=3.4225752464536923e-06, P172=2.831078469644628e-06, P173=1.6250695995312278e-06, P174=1.951603143052154e-06, P175=3.0040603620290026e-06, P176=3.350464475272197e-06, P177=3.176898840620173e-06, P178=1.7452992678056237e-06, P179=1.9830366601527924e-07),Row( P0=-593.3232798480326, P1=69.17169382950483, P2=-14.287865018191098, P3=5.884463988186754, P4=-7.17822565848075, P5=-15.754719748142577, P6=-1.7563568889329568, P7=1.1215476908262823, P8=0.026311200822622612, P9=-2.554490051162254, P10=-4.837013096657439, P11=0.5153330490705595, P12=-5.516712074135586, P13=-1.0607585537016837, P14=1.9689874309582136, P15=0.26487004905817296, P16=2.356963533515428, P17=0.9133656355736653, P18=1.3390003173658402, P19=4.486965347529349, P20=2.107372411173718, P21=4.145378450844069, P22=3.0760898437467947, P23=4.499407318743412, P24=5.087705056264455, P25=6.582893685525316, P26=3.2486879170723175, P27=7.349047607081474, P28=4.50354098415062, P29=6.57623316302068, P30=3.6994593984145685, P31=3.5033970852055147, P32=2.565354085420289, P33=1.7450787083405148, P34=0.5195230429264774, P35=3.4413003692065964, P36=-0.06863351955486358, P37=1.7535733783089504, P38=-0.3929077817574144, P39=0.9458217923624396, P40=0.6926808725341058, P41=0.6545557701168759, P42=0.6140440546980084, P43=0.6458800614871524, P44=0.6084530498264313, P45=0.6179085717310275, P46=0.6521877095428684, P47=0.6167837607240193, P48=0.5981998903873839, P49=0.617303303317375, P50=0.6496943651183326, P51=0.6623025842876749, P52=0.010255313487229847, P53=0.0007465315751991463, P54=2.4303063215896047e-05, P55=1.4188710047494212e-05, P56=1.6105438252156485e-05, P57=0.00016968234722741234, P58=0.0015983190752953214, P59=0.00805857758016329, P60=0.03878954665135432, P61=0.08267952263083606, P62=0.09102306904145407, P63=0.0688779091876451, P64=0.027267497381316216, P65=0.0118395159358833, P66=0.006125179668053389, P67=0.007356833070342249, P68=0.010662465385759063, P69=0.01893959547854157, P70=0.04122964752414327, P71=0.03240154716738118, P72=0.016943900526804327, P73=0.012915029598152977, P74=0.008259748315937638, P75=0.006017475994868065, P76=0.010372526704293073, P77=0.028307046142111776, P78=0.027867145210693806, P79=0.028713316915498464, P80=0.04747062146583649, P81=0.04632623251611669, P82=0.07206604347969989, P83=0.05558768100875904, P84=0.05403673309554159, P85=0.09050263480508268, P86=0.03686255417537324, P87=0.02790958483612695, P88=0.05026006630027556, P89=0.05439426965427127, P90=0.02892849239990468, P91=0.014101948253479239, P92=0.01540740893418357, P93=0.009579548131019194, P94=0.013098942791483756, P95=0.011049402765958232, P96=0.006658144188519115, P97=0.004935267292803107, P98=0.002478843951330256, P99=0.0009552208518679815, P100=0.002605340428657705, P101=0.005355865597789105, P102=0.0031589212930832598, P103=0.0009614898076187089, P104=0.0009934002749929797, P105=0.0025794266029187376, P106=0.0054557748204075605, P107=0.0025867634366661288, P108=0.001390737787888143, P109=0.0018843380520896676, P110=0.003816616679139697, P111=0.005219812966314852, P112=0.002073071007099861, P113=0.002814858916734598, P114=0.0036672588025779182, P115=0.003328321088832778, P116=0.004035127061287312, P117=0.0030360655728124478, P118=0.003318700771727651, P119=0.0024169930613244196, P120=0.004474281207888059, P121=0.0038009772763081015, P122=0.00239156498699163, P123=0.0030514327986395223, P124=0.00129518768632569, P125=0.001957134830287345, P126=0.0011645697902949103, P127=0.0006337625324927683, P128=0.0006308817828478665, P129=0.00041909054594016504, P130=0.0005903628889864724, P131=0.0007984110720312751, P132=0.0011832647243050572, P133=0.0006337057029010997, P134=0.0006485168469006252, P135=0.0005082440418760549, P136=0.000392078337309572, P137=0.0004170082857880701, P138=0.00035228635222969544, P139=0.000463980748441211, P140=0.0007612606023362383, P141=0.0005836912373364412, P142=0.0005399312872126937, P143=0.00017868843305357714, P144=0.00015646630853288638, P145=0.00016449382164934384, P146=0.0001666115282067872, P147=0.00019422925054016693, P148=0.0001530876429903224, P149=7.661877162777633e-05, P150=2.5190838403647696e-05, P151=2.027711831791791e-05, P152=3.533142956571498e-05, P153=5.5629304509077274e-05, P154=0.00013311670050164217, P155=0.0001054727227524948, P156=6.750832889884616e-05, P157=6.939842010367701e-05, P158=6.515523368065424e-05, P159=4.3443607931644245e-05, P160=4.5412258083357255e-05, P161=6.956400207839281e-05, P162=5.301275269772622e-05, P163=3.6978729702250044e-05, P164=4.543145660165938e-05, P165=3.3616830698393155e-05, P166=2.791398216089793e-05, P167=1.5557064353737824e-05, P168=1.2077565805912685e-05, P169=2.147602688224317e-05, P170=2.141479954634645e-05, P171=1.7525346190401533e-05, P172=1.531709170660797e-05, P173=1.3279815282125245e-05, P174=2.297902271663372e-05, P175=3.057855395249501e-05, P176=3.13871474882427e-05, P177=3.1866672397009385e-05, P178=2.0274640338305572e-05, P179=3.5244511267649193e-06),Row( P0=-632.1838477669515, P1=78.03227923014762, P2=-8.580728881833458, P3=14.009169277729756, P4=-5.162328994321058, P5=-3.882570861922817, P6=0.6576065256153109, P7=3.0072217864644344, P8=7.1720528641228976, P9=4.759494531348558, P10=3.877245595970751, P11=2.4444788194181237, P12=2.322099576728209, P13=4.7662031681632, P14=-7.552192549983171, P15=1.6670514514112627, P16=2.91640118958086, P17=1.7295710392979333, P18=5.603847429233823, P19=3.608497030393175, P20=1.4734830447483218, P21=0.1035809452115191, P22=2.9237296495250753, P23=-2.316858731647538, P24=-2.404039497646653, P25=2.733550265065448, P26=0.8695808345391762, P27=7.126867222573288, P28=3.6136300511623682, P29=9.116065302780502, P30=3.691270812926243, P31=5.314680269703043, P32=0.25539280975790773, P33=5.716067504533772, P34=5.109801007125479, P35=6.192478620905408, P36=2.906554721198406, P37=1.6306185547826304, P38=1.2569656120661883, P39=1.6213561811398403, P40=0.6101690889401855, P41=0.6367673835580754, P42=0.7241542943525954, P43=0.7460230099389104, P44=0.6764188497377301, P45=0.671533486128788, P46=0.6650649548135164, P47=0.6169997475034344, P48=0.6879067547591143, P49=0.6912829813044499, P50=0.7185183653001919, P51=0.67577060908427, P52=0.019518403150726473, P53=0.0010786084734707335, P54=7.299329826029476e-05, P55=2.116738400198352e-05, P56=3.804299536874201e-05, P57=0.0005102164750665442, P58=0.006902863014314644, P59=0.023576206928081486, P60=0.044981875879169735, P61=0.06025790368070607, P62=0.027601162330022556, P63=0.0036457472358004837, P64=0.0006816444534752117, P65=0.00235070996127853, P66=0.010391014669393872, P67=0.020128603558223938, P68=0.01553190477577213, P69=0.0169335873181314, P70=0.01625150279011447, P71=0.010610266095826498, P72=0.005208173526342755, P73=0.0025146811852737256, P74=0.002738299027513542, P75=0.004165160638343423, P76=0.0039232403342264075, P77=0.010108151425649659, P78=0.013404860508705384, P79=0.016200741161832306, P80=0.021677960836005917, P81=0.027582825768645684, P82=0.022799757361313837, P83=0.006748986899131362, P84=0.004568606533925429, P85=0.006787679256135456, P86=0.0044513421495868954, P87=0.004208851491604423, P88=0.006799439244783333, P89=0.007304992173390118, P90=0.004764487905604887, P91=0.0015772028177352914, P92=0.0011325926139501786, P93=0.0038068159698833445, P94=0.013052526921030712, P95=0.007549120611229757, P96=0.0019942612869321918, P97=0.0016195421985613476, P98=0.0014523776889524705, P99=0.0023210534956427872, P100=0.0007496483651512983, P101=0.00022450883188015572, P102=0.00025369650069487714, P103=0.0009288804849665301, P104=0.001549344372172536, P105=0.0010100700488872437, P106=0.0007173096299712846, P107=0.0004819478829623742, P108=0.0011121044268205067, P109=0.0008696492841677561, P110=0.0008673008296385302, P111=0.0015841843841281405, P112=0.0008699915720747654, P113=0.00035375861022096113, P114=0.0003164158119254422, P115=0.0004017613530986725, P116=0.0004186427059749372, P117=0.00029003563865765076, P118=0.0002523230593884377, P119=0.00024124949121651247, P120=0.0002961670572675072, P121=0.0002422218342963751, P122=0.00023345089242391874, P123=0.00019760332208030635, P124=0.0004603575571740731, P125=0.00033697724381939314, P126=0.00024208626485103756, P127=0.00020539144884474032, P128=0.00015825973408702284, P129=0.00014321864549732635, P130=0.00012707551911815315, P131=0.00016697507171321122, P132=0.00016099686781381506, P133=0.00025821733873156497, P134=0.0003701699441517749, P135=0.0004989150992214524, P136=0.00033609530508071927, P137=0.00019472963833764193, P138=8.313408099809828e-05, P139=5.557886362325048e-05, P140=3.232178304095194e-05, P141=3.84875346213139e-05, P142=3.396572667818535e-05, P143=4.194761523625038e-05, P144=4.0258055589509056e-05, P145=4.890200343219336e-05, P146=5.204522476876153e-05, P147=6.523536143671844e-05, P148=7.433560564350934e-05, P149=4.095919389192144e-05, P150=4.0316049276830546e-05, P151=7.65634070485768e-05, P152=6.014867432709621e-05, P153=7.505837630117189e-05, P154=0.00011338507847367817, P155=8.009314298057143e-05, P156=9.118962911954052e-05, P157=0.0001258179312565434, P158=0.00012578707407060943, P159=0.00010490554204752758, P160=7.689108731431223e-05, P161=5.8124469916175865e-05, P162=5.232005296930536e-05, P163=6.412928859789245e-05, P164=4.1301644547341704e-05, P165=3.684912183336451e-05, P166=5.972418949322502e-05, P167=6.614909725156769e-05, P168=4.303707881986217e-05, P169=3.207256649352004e-05, P170=2.2948544780013497e-05, P171=1.70017821464635e-05, P172=1.2290317107667105e-05, P173=1.9261449572541728e-05, P174=1.916557765564159e-05, P175=2.1740207702134936e-05, P176=2.466115909577329e-05, P177=2.1266881852293177e-05, P178=1.1010690459768088e-05, P179=1.1878311017281164e-06),Row( P0=-668.152803532011, P1=73.8205802222068, P2=-3.4939239781201783, P3=11.749678740292694, P4=-2.738139435438358, P5=-2.468920169870199, P6=2.706900318111461, P7=2.0083413810552067, P8=4.126664347654492, P9=1.1180495063863698, P10=2.2806101669361403, P11=-0.18862231577626182, P12=0.09354358857486653, P13=1.7590109141098182, P14=-5.108409960729332, P15=1.111551897536396, P16=-1.0225065036668923, P17=-0.9469030642779177, P18=2.838762427862686, P19=2.315455868035607, P20=1.5205541673782215, P21=0.41486379939921153, P22=3.440685501530116, P23=1.2509536142557938, P24=-0.19123582826527621, P25=2.6675386258085885, P26=-1.371940151298235, P27=1.8552185875160645, P28=-1.937827610655828, P29=2.095488792320926, P30=-0.038373865086850006, P31=2.766851575265572, P32=0.813237518511735, P33=5.5674261845883795, P34=2.2077198762808075, P35=4.53276656311559, P36=0.8723497552089998, P37=0.8269920927671914, P38=1.6586451865094725, P39=1.6275525816538101, P40=0.6769761642939246, P41=0.6220261275127326, P42=0.6659243405860875, P43=0.6928439255902857, P44=0.6685192217351471, P45=0.6739235991934466, P46=0.6938342924397691, P47=0.6787129525185812, P48=0.687310557029992, P49=0.6826883734892447, P50=0.6847869989084591, P51=0.6943786722055356, P52=0.0015324913684599839, P53=0.00017363100057770598, P54=5.558786797343339e-05, P55=2.740695622000035e-05, P56=6.59000157677037e-05, P57=0.001121998747298157, P58=0.00675433346668126, P59=0.016688197183492914, P60=0.017413867657532896, P61=0.02116310706737768, P62=0.0316635668872691, P63=0.023584616450339425, P64=0.013444047958010297, P65=0.008964955868611355, P66=0.011692775242555886, P67=0.01471111824251143, P68=0.016282519487025866, P69=0.01145826739943475, P70=0.005263865767752184, P71=0.0023172054796664833, P72=0.005468764626526973, P73=0.006667299479685463, P74=0.006825532615087233, P75=0.013631866878818098, P76=0.016768185437409086, P77=0.025244656159352848, P78=0.01900546459422258, P79=0.01015161634043934, P80=0.004789424914207536, P81=0.003736658116338253, P82=0.005659285917390927, P83=0.018042513499456715, P84=0.029223383120557927, P85=0.01375689389963017, P86=0.0026702373056578257, P87=0.005990684295534756, P88=0.005565131660728854, P89=0.0023751931073900585, P90=0.0019043123580206143, P91=0.005333087598655647, P92=0.0371100485702829, P93=0.011953774130717403, P94=0.0009583012350527316, P95=0.0006774156178180476, P96=0.0010578923699076789, P97=0.0013088786927991749, P98=0.003118473946389542, P99=0.01233540785214149, P100=0.0023433475978346203, P101=0.000618565611214905, P102=0.0004348220900768108, P103=0.00036074971124065675, P104=0.0033899993440370526, P105=0.0057707638787270745, P106=0.000985166791301944, P107=0.00013206899350907573, P108=0.00035065011050266923, P109=0.007171244475057661, P110=0.004351231446303853, P111=0.00030243213225415977, P112=0.000161954212803866, P113=0.0006177443028145095, P114=0.0008000927728059645, P115=0.0001773580799833097, P116=0.00022090127117589762, P117=0.0006593686774062815, P118=0.0002508054066742825, P119=0.00022943781337333684, P120=0.00032742964147292394, P121=0.0003836862769222196, P122=0.0001294469678870412, P123=0.00012458203737063155, P124=0.00012405672980840438, P125=6.045331469531093e-05, P126=6.510321242320872e-05, P127=7.164823020617334e-05, P128=3.9143027167741026e-05, P129=6.260571361547032e-05, P130=9.322895823780257e-05, P131=0.00010097777175847579, P132=0.00011232840172182772, P133=0.00011951467462880152, P134=0.00018840466167281773, P135=0.0002494913083885336, P136=0.00010678679593364377, P137=0.00011100680451893975, P138=5.4440945790445175e-05, P139=5.4692196519767924e-05, P140=5.1392785764534546e-05, P141=4.407796159542533e-05, P142=3.6205325726911165e-05, P143=4.447422668998711e-05, P144=5.9146755052652955e-05, P145=5.0472410595991445e-05, P146=4.748431623567207e-05, P147=4.529675734255136e-05, P148=2.879074416439971e-05, P149=2.140886976228621e-05, P150=1.8065871102119857e-05, P151=2.0822567269186094e-05, P152=3.837673077827392e-05, P153=4.4164451740700734e-05, P154=3.5385551411132786e-05, P155=2.5890725709261108e-05, P156=3.249329141248434e-05, P157=5.722586090250391e-05, P158=5.7407091087498764e-05, P159=2.108466029309602e-05, P160=4.0485852859472006e-05, P161=1.4261108430094416e-05, P162=1.1225880272246603e-05, P163=7.5707792818517316e-06, P164=1.3402089100111253e-05, P165=1.1921013486240399e-05, P166=9.086530525036565e-06, P167=1.3465760165171053e-05, P168=8.564229122316729e-06, P169=4.807517662970395e-06, P170=4.306577432278618e-06, P171=4.325428006477382e-06, P172=4.703557599784188e-06, P173=4.299060047326e-06, P174=4.6132088593205505e-06, P175=4.213291708724763e-06, P176=5.190875170111459e-06, P177=4.8131229404108215e-06, P178=2.7741297590813125e-06, P179=3.9632348862793156e-07),Row( P0=-664.8123392754891, P1=75.14264099537154, P2=-0.25656847822258233, P3=9.35539478326433, P4=-0.9906606458003725, P5=-4.514265768102442, P6=1.914180229981398, P7=1.2087622797194368, P8=2.7123206983223285, P9=4.315561632997136, P10=0.7298395616142307, P11=-0.5947391233165984, P12=-0.24137896163570785, P13=2.5814273154662737, P14=-3.8324349659695414, P15=1.5929583760214878, P16=0.17559092753660602, P17=0.022793667698694783, P18=3.0404997602174393, P19=0.7644022582683383, P20=-0.1460444558088466, P21=-0.0651240100184823, P22=-0.08394525209159563, P23=-1.2880672781677833, P24=-2.5787598720572382, P25=0.8703142139833269, P26=-1.1883598761636103, P27=3.4541812390318993, P28=1.235333963551054, P29=4.486536688513937, P30=2.0733376225265996, P31=4.000391836154285, P32=1.3926890899650617, P33=5.120521009362149, P34=3.984831266986849, P35=5.416757388598174, P36=3.225873331791814, P37=1.724542723003044, P38=0.728129754376752, P39=0.7781904125992865, P40=0.7102074005479183, P41=0.7057319489817576, P42=0.7396718796396472, P43=0.7062008989728589, P44=0.7067486407121053, P45=0.7003895818334774, P46=0.6734131287720981, P47=0.6536800010417728, P48=0.7048940475240405, P49=0.7331593859737646, P50=0.7716176825383356, P51=0.7486299294522722, P52=0.00012523755759837093, P53=9.999903166491146e-05, P54=4.9893664196901165e-05, P55=3.25169876868013e-05, P56=5.839259761652846e-05, P57=0.0008472925506946288, P58=0.008569087060817621, P59=0.0210379312168464, P60=0.02810901415975653, P61=0.033923037735511155, P62=0.02216047421100814, P63=0.005331502262024374, P64=0.0011189567989459315, P65=0.004823049023771865, P66=0.017570343765733864, P67=0.02400365989757042, P68=0.016607735653015203, P69=0.014381839308078587, P70=0.010911005158156557, P71=0.005823436108628468, P72=0.00733465764523125, P73=0.006602462862093596, P74=0.005115984721670515, P75=0.005287285475652983, P76=0.005651632017494682, P77=0.012874364713525502, P78=0.016209520446917272, P79=0.01757847379879975, P80=0.012888965306060457, P81=0.01457387696650962, P82=0.027647730085966632, P83=0.02529438402651521, P84=0.023510573540460343, P85=0.025135113257796163, P86=0.01082345513049941, P87=0.008670408767404316, P88=0.012146605036008683, P89=0.004437257077488715, P90=0.0017826296231137633, P91=0.007903386855862645, P92=0.02273202863630172, P93=0.012455169723768254, P94=0.007866479453681678, P95=0.0038243504194815656, P96=0.002786052885124685, P97=0.018253640547278495, P98=0.01286997279796508, P99=0.0017513380032379343, P100=0.00037667207918171564, P101=0.0004916017684254002, P102=0.0015639375188001685, P103=0.001462036609304841, P104=0.0010833164135415601, P105=0.0015860087489939576, P106=0.004853303338990802, P107=0.001848391444044756, P108=0.0018177437951176208, P109=0.002860762740755975, P110=0.0021980119901602677, P111=0.001321950090207088, P112=0.0007130099224030028, P113=0.0005892703538194077, P114=0.0005524813524191116, P115=0.0004760099911224412, P116=0.0007774999314345093, P117=0.0005120819679289522, P118=0.00020356385609448822, P119=0.00036511520938345936, P120=0.00047521309125367556, P121=0.0002558422120262646, P122=0.00029146347248535216, P123=0.00017594768548895014, P124=0.0002803771472698293, P125=0.00025744967185947006, P126=0.00017853596673097228, P127=0.00021969788287525373, P128=0.0002468220799915586, P129=0.00014470988711677224, P130=0.0001409029153018079, P131=0.00021277388061037117, P132=0.000242562312848123, P133=0.000279075830622057, P134=0.0004861241668711732, P135=0.00044146046447213574, P136=0.00024436228809372274, P137=0.00024938514340670223, P138=0.00014357611172153458, P139=8.589471217104707e-05, P140=5.675454267810109e-05, P141=4.3315450544776015e-05, P142=3.9246913807114345e-05, P143=3.7709103318571886e-05, P144=4.3966052612229534e-05, P145=4.66354755218685e-05, P146=4.080317606358006e-05, P147=3.752171575956894e-05, P148=4.4030185134987646e-05, P149=4.3828307822065954e-05, P150=5.140889918603394e-05, P151=8.825907741444827e-05, P152=6.534723093610268e-05, P153=9.096654327755101e-05, P154=0.00010967918333660851, P155=7.756448621473096e-05, P156=8.370527102403138e-05, P157=6.690079230424642e-05, P158=5.0777528175666396e-05, P159=4.4478361889015835e-05, P160=4.888009859778989e-05, P161=2.7198149263353403e-05, P162=2.8603463981258478e-05, P163=5.970593604883009e-05, P164=5.600825214421406e-05, P165=5.95838981944866e-05, P166=5.629579064298283e-05, P167=3.7437529476053267e-05, P168=2.964615983931913e-05, P169=2.4125606228154014e-05, P170=2.4267935801161344e-05, P171=1.907192472745305e-05, P172=1.8010180768591504e-05, P173=1.746264135566826e-05, P174=2.391364182464471e-05, P175=2.943044299627228e-05, P176=2.543460918850109e-05, P177=2.9792825648048603e-05, P178=1.6626169501014478e-05, P179=2.0194730005754875e-06),Row( P0=-543.4279833617443, P1=48.282615729578744, P2=-4.8325209853795945, P3=12.487920292441673, P4=-2.7333899637669337, P5=-8.262657954969832, P6=5.539355151141286, P7=3.016430030540276, P8=3.933302534417332, P9=4.350716589934096, P10=0.06751956709912947, P11=4.414191425603384, P12=-1.8572992666074324, P13=1.8549629354381636, P14=2.6528619491286007, P15=2.2368110113076054, P16=3.9384125681203894, P17=4.708692984339598, P18=6.846095436369686, P19=7.357404224721635, P20=7.2832891869447165, P21=6.982984109808302, P22=4.111163903471633, P23=3.608469162674865, P24=1.9991364479623392, P25=7.650139927244308, P26=7.3688608144850765, P27=10.165026569117568, P28=7.427361399390062, P29=7.744061706437568, P30=2.6501044772614457, P31=3.6225825308918966, P32=0.16715967541972507, P33=2.025894777756957, P34=-2.6363225475250234, P35=2.0548347726656817, P36=0.49047296601611595, P37=0.9095959744585499, P38=-0.37445597954976073, P39=0.46291139553729477, P40=0.6352912545499815, P41=0.6276065696239816, P42=0.7150419289465958, P43=0.7597131602789509, P44=0.706861673998163, P45=0.7078594184713191, P46=0.7169107263582087, P47=0.6897281763186468, P48=0.6893021003870352, P49=0.7666721964193127, P50=0.7593319221611206, P51=0.7163121309944644, P52=0.05765120765853335, P53=0.01411646134809004, P54=0.0008351294077692355, P55=0.0001143173649971091, P56=0.00011258241659880828, P57=0.00019757308591935034, P58=0.0003367206927278793, P59=0.008325625687344853, P60=0.08271484719607237, P61=0.16325068115682898, P62=0.41705863818357847, P63=0.5627407645816923, P64=0.23678517533230278, P65=0.36032912807155154, P66=0.20236352846597544, P67=0.0391277446763217, P68=0.021962040888255475, P69=0.07694259105262906, P70=0.09567259602667974, P71=0.07741775075326793, P72=0.073482410721169, P73=0.06430882961715684, P74=0.0884321496297, P75=0.09570720945316322, P76=0.04308207023979249, P77=0.09476041177574365, P78=0.19502665233306876, P79=0.29724912629978256, P80=0.22784424920681, P81=0.16112655295302528, P82=0.14528231854809134, P83=0.07710538867908595, P84=0.06293515712984579, P85=0.0581385280059222, P86=0.04707205333741659, P87=0.14939032087410828, P88=0.16412492920382502, P89=0.13989362610448194, P90=0.1686730819694885, P91=0.20665578708145385, P92=0.48820668862018307, P93=0.19259599996172136, P94=0.07239111578830217, P95=0.023935409197398467, P96=0.010942420263677963, P97=0.02142850889356868, P98=0.06523664198503303, P99=0.06061004377433179, P100=0.05116335719110075, P101=0.0532064470287505, P102=0.010676564013190892, P103=0.003259422706443036, P104=0.007215342991984702, P105=0.011078644673822694, P106=0.009631122020788658, P107=0.01831581634587451, P108=0.04476364621483328, P109=0.02675496743716817, P110=0.03395927850689536, P111=0.021907192187937883, P112=0.01645051417602347, P113=0.03238566391890455, P114=0.03645661002450307, P115=0.015459565116642591, P116=0.015484167640223631, P117=0.016017374498461103, P118=0.008512908120138938, P119=0.011828555857291013, P120=0.006608948994846584, P121=0.020817966230987996, P122=0.008565313944593173, P123=0.004361923446157655, P124=0.005781823696763092, P125=0.004533614754600621, P126=0.003792786321293351, P127=0.005762178031993911, P128=0.0053215115013302595, P129=0.004994749119614951, P130=0.010565128531529792, P131=0.015917036307539253, P132=0.014359947223965481, P133=0.008105723576268611, P134=0.007900295462645337, P135=0.006586199779053021, P136=0.004947818685637512, P137=0.004324550450123812, P138=0.004007076925170396, P139=0.0051406027029094695, P140=0.006109319084211817, P141=0.004474769942603239, P142=0.0028894182198269305, P143=0.0023846652319130957, P144=0.00226443603760249, P145=0.00179974329310976, P146=0.0020224424496810066, P147=0.0009757231317276566, P148=0.0009860614258523001, P149=0.000677222825203099, P150=0.0005304471319786256, P151=0.0010396712662858912, P152=0.0015801178055062268, P153=0.0016390774118200425, P154=0.0013026295623070265, P155=0.0013160445755561112, P156=0.001164074733049841, P157=0.001095906635354129, P158=0.0008139790791544914, P159=0.0006963964266032544, P160=0.000990743627383655, P161=0.0008120376602281402, P162=0.0006133982624532268, P163=0.00043419206008450017, P164=0.00025081273290174336, P165=0.0002052313233433116, P166=0.00023109143776632734, P167=0.0003452981125073018, P168=0.0004314712275070283, P169=0.00041974130582335325, P170=0.00042652473593006684, P171=0.0003756253764238233, P172=0.00043511475278674804, P173=0.0005071567817005033, P174=0.000495655392308155, P175=0.000550143473034254, P176=0.0007567328670376072, P177=0.0009342597705849008, P178=0.0006008670886863878, P179=0.00010731049353377261),Row( P0=-517.4187248468896, P1=47.78831681026384, P2=-4.915062443363812, P3=11.22984935430979, P4=-2.0501237133303936, P5=-5.496162550948709, P6=4.266647150463853, P7=0.950665919165684, P8=2.397146900924984, P9=3.8798199753909564, P10=-1.643238197424568, P11=2.943660879932062, P12=-0.8362870775870209, P13=1.6260009545357625, P14=1.8258284876381525, P15=0.2879599124980304, P16=2.1382088151808825, P17=2.1900131563926357, P18=5.604635546689628, P19=10.229844734873064, P20=9.915696852332006, P21=13.273975026045678, P22=11.113326545841243, P23=8.951067216374557, P24=3.3046899134113694, P25=4.43772426676851, P26=1.569994814812623, P27=4.235247933820293, P28=4.3850847034389, P29=5.082750187608002, P30=1.4660657074221892, P31=1.3831431939126457, P32=-2.501918339099579, P33=0.848392827402196, P34=-2.9430699119035575, P35=1.9293711893946868, P36=-0.323364373263655, P37=-0.8906005732086426, P38=-1.178182862408094, P39=0.5215945312347786, P40=0.649527281584482, P41=0.6562668336414902, P42=0.7199801852977937, P43=0.738150884796652, P44=0.6739015004931345, P45=0.6774478878782841, P46=0.7436879606178718, P47=0.7911051300716063, P48=0.7396260306682358, P49=0.7381109625552071, P50=0.7234677252904106, P51=0.6930066813225995, P52=1.099261995535228, P53=0.13337707148318836, P54=0.006394744980696315, P55=0.0019082997524386314, P56=0.0010729331675048937, P57=0.0009204788261615958, P58=0.0006920718421342229, P59=0.0041559076086227255, P60=0.04205878766824513, P61=0.11861833728129018, P62=0.4589374543270992, P63=1.0042696031016662, P64=0.6078845661245444, P65=0.35777033128571056, P66=0.09348813570923525, P67=0.01571543294664297, P68=0.007736147279403598, P69=0.009427224469132297, P70=0.0136537560380218, P71=0.011133145335217486, P72=0.018047789141344747, P73=0.0282996697689973, P74=0.18848926949253222, P75=0.49034914625426196, P76=0.3657026138070803, P77=0.35483290599359574, P78=0.4935072698920426, P79=0.4272852647040608, P80=0.10614017933913307, P81=0.016393862513865123, P82=0.012135349552087336, P83=0.02161177435818508, P84=0.05441859052570357, P85=0.09858870839802647, P86=0.3456894048723261, P87=0.7109415312610223, P88=0.8062770184008721, P89=0.30819303618706057, P90=0.21102871394251566, P91=0.38763556317675296, P92=0.18122915928834005, P93=0.03910938149956411, P94=0.07088644649968674, P95=0.14244519016722165, P96=0.2662891136300794, P97=0.28787101454447145, P98=0.06866339546635619, P99=0.041115671698859, P100=0.14443439706900488, P101=0.06799449238007929, P102=0.035198901081326166, P103=0.05834238482217037, P104=0.036077097814390696, P105=0.03992173962851121, P106=0.07035083997217165, P107=0.049531555160071544, P108=0.05860757789726699, P109=0.14025981421447822, P110=0.07480405848348017, P111=0.0277178336980917, P112=0.050238251759192186, P113=0.055758770406224865, P114=0.18101577063600505, P115=0.038883583760827775, P116=0.008285363024930485, P117=0.008561994109597352, P118=0.04330666002834553, P119=0.027287231346625602, P120=0.004588298835580819, P121=0.010639825818083605, P122=0.02035158289812346, P123=0.013969032973512393, P124=0.009980426669106366, P125=0.012003334335101538, P126=0.0034771823368403188, P127=0.005465272950502153, P128=0.008478662610347363, P129=0.005840686898806817, P130=0.022669126980832226, P131=0.0138223843528622, P132=0.021268842182872713, P133=0.024181170693840546, P134=0.010143602877513523, P135=0.020714139223768937, P136=0.007150459481354644, P137=0.005817457828692639, P138=0.0032674577040447622, P139=0.004818126631377475, P140=0.002396771559796685, P141=0.0027027164035685718, P142=0.0018758812314421222, P143=0.0022540785869310143, P144=0.002689548563825397, P145=0.0015279979496158624, P146=0.0011255366661989815, P147=0.0012694521660899564, P148=0.000707077178894072, P149=0.0003095448723782471, P150=0.0007316602675304036, P151=0.0011266114380009996, P152=0.0008909953038480398, P153=0.0004563168129790769, P154=0.00048382590127013536, P155=0.0003400250154307977, P156=0.0005275660496829675, P157=0.0007081685241641495, P158=0.0003791834259497109, P159=0.0005217873940229134, P160=0.0005456078948501767, P161=0.0007168480955584744, P162=0.0003276394608627764, P163=0.00019084243061462272, P164=0.00012050997954010461, P165=0.00027965043319000947, P166=0.0005916743923156648, P167=0.00034379756437911263, P168=0.00019791445061434368, P169=0.00014688386263912346, P170=0.0001615898819745297, P171=0.00014928550818688743, P172=0.0002188197743230853, P173=0.0002321005371990825, P174=0.0003538219561492667, P175=0.0002978402745012354, P176=0.00030538177950348126, P177=0.0003860253872125706, P178=0.00017829899359352613, P179=1.784690310608168e-05),Row( P0=-652.1547601220313, P1=81.59457249048636, P2=-1.095406151131386, P3=11.032545285352693, P4=-1.2100031574849701, P5=-0.9325037994837229, P6=4.116580524532877, P7=1.5419835326517841, P8=2.0906774305673497, P9=2.6294950456076958, P10=0.05053665477137224, P11=-0.7590600514873838, P12=1.0744895380841346, P13=1.6256277251414926, P14=-1.741820540892853, P15=2.5528367521236977, P16=0.47335836711986334, P17=0.7290372017464151, P18=2.9917504075272165, P19=1.3480676153503284, P20=-0.37799523925058603, P21=-1.8734074596060233, P22=1.7441764674442142, P23=-0.8028860287673418, P24=-0.38460450079477415, P25=1.389379487134815, P26=-0.9440682432220472, P27=2.5345020848885462, P28=-0.09310992136613533, P29=2.4339802409710387, P30=2.5302014869363103, P31=5.454751520817289, P32=2.866510282018077, P33=6.210131558623366, P34=4.565697260719694, P35=5.955562899141637, P36=3.681669382274154, P37=3.5168304078681705, P38=2.6212578558921145, P39=1.7463036309239803, P40=0.7327424628186656, P41=0.6898687940178503, P42=0.681470474238011, P43=0.6925099049962468, P44=0.6894265058500814, P45=0.7073385386206711, P46=0.6920576721079595, P47=0.6631400551563692, P48=0.711650706184378, P49=0.7173146801829061, P50=0.7403279445703289, P51=0.76437845860738, P52=0.0011063519026633373, P53=0.00023331536211709336, P54=9.472365099276726e-05, P55=2.5006923865531207e-05, P56=2.1558283186977268e-05, P57=0.000599484142518417, P58=0.006213064441508038, P59=0.015208910860331503, P60=0.01819400556857897, P61=0.021937521643496546, P62=0.02016036390392031, P63=0.004612016634179302, P64=0.0010196909699440675, P65=0.005575029194635065, P66=0.014957544870736635, P67=0.01900378139177642, P68=0.010459051976555013, P69=0.008767325838149693, P70=0.00946383294304168, P71=0.007465768288249794, P72=0.015352802540870824, P73=0.020523286508760896, P74=0.008060047421729177, P75=0.004000589885923596, P76=0.002327935029498963, P77=0.004332717042727137, P78=0.005490382303501486, P79=0.009222059065137126, P80=0.011439520322823209, P81=0.010415320587877698, P82=0.018058797183075008, P83=0.048451132647431706, P84=0.04958992243153449, P85=0.014702278874905633, P86=0.0038824617066143697, P87=0.0022161088335686833, P88=0.004355087487141744, P89=0.0049508140187023925, P90=0.002761145189912958, P91=0.0014455173977545022, P92=0.018722812004330156, P93=0.01448903380097853, P94=0.009840550571601067, P95=0.013130872246686645, P96=0.002918046399625439, P97=0.010031221273520245, P98=0.048537252302158815, P99=0.011965815963479716, P100=0.0014372761658780558, P101=0.00047438008315751716, P102=0.0010484881619571668, P103=0.0033805553083961467, P104=0.002589314404134941, P105=0.0005615067435800969, P106=0.0012545257725799183, P107=0.004901533006560318, P108=0.0019474733590119165, P109=0.0004731008720681767, P110=0.004419077211903468, P111=0.00498776296895943, P112=0.00031738812340457095, P113=0.0008358469714700337, P114=0.0014795904721452828, P115=0.00020332559048572439, P116=0.0006726363734163235, P117=0.0003501345152204052, P118=0.0002032838400797932, P119=0.00028982188105431317, P120=0.00045505726729451435, P121=0.00046063124531503546, P122=0.00010901809797769772, P123=0.00043944769594842577, P124=0.00043717657162255376, P125=0.00016364949345840815, P126=0.0003044891849957484, P127=8.096777373794017e-05, P128=8.110931623033166e-05, P129=8.976840257257398e-05, P130=7.134172916666179e-05, P131=0.00014166222309773254, P132=8.592105432034472e-05, P133=0.0003288379598258958, P134=0.0002758308043561277, P135=0.000543835907753726, P136=0.0002704652422139122, P137=0.00017292628048063373, P138=8.965223330444961e-05, P139=8.417852280363131e-05, P140=5.642082410619028e-05, P141=5.320858823908583e-05, P142=5.22892086183019e-05, P143=5.7120129892349645e-05, P144=4.921243860277689e-05, P145=3.47125819650656e-05, P146=3.521554948838051e-05, P147=3.3857331641192974e-05, P148=2.4293784572814427e-05, P149=2.809198478741984e-05, P150=1.432420387168811e-05, P151=1.9395053012118373e-05, P152=2.169314513223781e-05, P153=2.567921706323806e-05, P154=3.561865185564954e-05, P155=3.736611385214898e-05, P156=2.466604435089685e-05, P157=2.5005737579053435e-05, P158=2.2482938801214457e-05, P159=1.4320596304825833e-05, P160=1.0542824739233545e-05, P161=8.897937384412475e-06, P162=4.75872463314583e-06, P163=5.673801882861238e-06, P164=8.36195916120344e-06, P165=1.1008820911470342e-05, P166=1.2439161237933833e-05, P167=1.3162834842335269e-05, P168=8.911642285726252e-06, P169=5.092455785406071e-06, P170=2.739423521763285e-06, P171=3.986660263746644e-06, P172=2.3528796958546864e-06, P173=2.5075870426210396e-06, P174=3.5759394658193924e-06, P175=2.896113486801662e-06, P176=4.030924103167418e-06, P177=3.4404429032428876e-06, P178=2.579229759689639e-06, P179=3.024995880820246e-07),Row( P0=-609.0315215922863, P1=73.63041333294811, P2=-15.559081390518033, P3=9.065622990166625, P4=-2.8072119227364345, P5=-12.3878780718437, P6=1.05486948220674, P7=8.651348941949673, P8=7.049979483204896, P9=-4.340497941321959, P10=-2.20277045577256, P11=4.0888253064168865, P12=-1.5254060749802245, P13=-2.5128614463479684, P14=3.013750915192333, P15=6.352790054193574, P16=1.8542730208381217, P17=0.6372009650531582, P18=4.438310087930461, P19=5.248100415798445, P20=2.573700063803729, P21=4.679597262607961, P22=4.082564436721311, P23=-0.3780595161443945, P24=1.6038891381638094, P25=2.4889049527186806, P26=-0.9583919589236263, P27=2.2197831003479105, P28=0.14468279969445824, P29=4.295662310991931, P30=1.10634739414647, P31=1.718012258361927, P32=1.236092241951094, P33=2.7668476288678323, P34=-0.2397380065843027, P35=4.504002615205401, P36=1.3824964714895742, P37=2.28269410927368, P38=-0.028013499589201418, P39=1.6217610924986325, P40=0.6756996212403914, P41=0.6017516410696951, P42=0.5442096045814102, P43=0.5787670639988772, P44=0.5677284977050491, P45=0.57732614774231, P46=0.6142840235958317, P47=0.6359860729020629, P48=0.6057470102741187, P49=0.6933766053966552, P50=0.7302009693893661, P51=0.7225312658096538, P52=0.00022979307381623438, P53=0.0004084389364472272, P54=0.00016329040365229544, P55=3.4230986133322105e-05, P56=4.265603270210558e-05, P57=0.0009524489104220648, P58=0.006816702990553879, P59=0.013439047785009884, P60=0.018459010763978535, P61=0.0501929713063505, P62=0.08673915475968173, P63=0.11341656028128337, P64=0.04226364675563469, P65=0.015181718795574686, P66=0.041849909952883936, P67=0.04397381501311124, P68=0.016407922721266517, P69=0.007638502860935954, P70=0.011504728924293446, P71=0.013973300890606756, P72=0.022133459130726757, P73=0.02311488393760825, P74=0.020537458730073396, P75=0.025422966075029193, P76=0.022791547104945874, P77=0.016541468903570857, P78=0.012156887250174195, P79=0.01855929996635357, P80=0.018131444744615807, P81=0.02947558555454436, P82=0.06070773103324495, P83=0.05702091439878012, P84=0.04773481962364924, P85=0.03873254729921162, P86=0.02173790462112164, P87=0.011711259116297416, P88=0.03922133199046109, P89=0.029558828426843656, P90=0.01281927976276329, P91=0.00513290360631918, P92=0.005211807289046765, P93=0.010819408361507755, P94=0.018534556241486574, P95=0.01514222183565298, P96=0.004563794959573023, P97=0.003696437588535896, P98=0.002552134154808806, P99=0.0017667432604980401, P100=0.0010947051926678006, P101=0.0021985220241341, P102=0.002718816858620429, P103=0.0020473376637846607, P104=0.0021074721068062856, P105=0.0008052226327099315, P106=0.0011146767751041978, P107=0.0012491809273867042, P108=0.0008310199984511058, P109=0.001683083646069309, P110=0.0014847656865551633, P111=0.0016321058939372083, P112=0.0015133743385885897, P113=0.0019801445855684037, P114=0.002049696683146442, P115=0.002886193095717206, P116=0.002870607998640243, P117=0.00554996877411159, P118=0.009065142165776472, P119=0.0038471618000264834, P120=0.00904318059715854, P121=0.015334505814385736, P122=0.004424219305246097, P123=0.00308473455004836, P124=0.0031172776385577543, P125=0.00353253533483675, P126=0.0011359361600234796, P127=0.0007257257112657433, P128=0.0009455241811029039, P129=0.0008361375394185112, P130=0.0007469569920902291, P131=0.0009854257042429585, P132=0.0023209634125344845, P133=0.0011162436898797072, P134=0.001991566265937662, P135=0.003644671168012504, P136=0.002065928078581851, P137=0.0015097201319548576, P138=0.0008037406119833157, P139=0.0006058233426972067, P140=0.00039681997660714197, P141=0.00021336031728980316, P142=0.0002885634636690667, P143=0.00043415942781832036, P144=0.0002936730665266142, P145=0.00017400437032645815, P146=0.00023666284328415364, P147=0.0002156659585209589, P148=0.0001771409043662883, P149=9.476143080244993e-05, P150=6.096800607930577e-05, P151=6.94166256458248e-05, P152=0.00010043333560341468, P153=0.00012942369818497133, P154=0.00014762247788790086, P155=0.00019045810797500392, P156=0.00016109215872837904, P157=0.00013818742503373302, P158=0.0001744024124217845, P159=0.0002169073164985459, P160=0.0001896756933880417, P161=0.00016874057445395767, P162=0.00013481707425365893, P163=0.0001492903627759758, P164=0.00011098499136685458, P165=8.273082613117598e-05, P166=0.0001278028481917643, P167=9.009396393190914e-05, P168=0.00014374608947047334, P169=9.76937448255363e-05, P170=4.2692470599465637e-05, P171=8.431309962060817e-05, P172=5.091227402854009e-05, P173=6.844931933044263e-05, P174=0.00010899354976419708, P175=0.00012449025351510127, P176=0.00013015467984205708, P177=0.00011597490778789075, P178=5.569658889457519e-05, P179=6.700161433460502e-06),Row( P0=-623.398093037087, P1=57.255551527809516, P2=1.6105329299616018, P3=12.877912168890125, P4=-0.8546363220869369, P5=0.4790516603652839, P6=1.580966886685528, P7=1.4081703720029588, P8=2.9608565180829727, P9=0.7448011051724424, P10=-3.800472172947051, P11=2.414860562088782, P12=-2.331065225243765, P13=0.42772724254581485, P14=2.3304167476412214, P15=-0.08118631924312317, P16=1.8015423488163587, P17=1.2801092259042919, P18=-1.1503740966827394, P19=0.9396714089168092, P20=1.0342082956419083, P21=-0.7578908150248771, P22=2.554765811606632, P23=2.5873073408689655, P24=0.46711081895695455, P25=4.166444813788574, P26=2.1719841843520498, P27=4.989159083862619, P28=2.699606168774135, P29=4.107170458732004, P30=2.395329964396755, P31=4.109123460209515, P32=1.6649830278760989, P33=3.4656339779838565, P34=-1.2358952972729713, P35=2.0563483034018804, P36=2.058829927006536, P37=3.332114694519435, P38=1.3936319271857844, P39=2.3497634648696115, P40=0.7022919481322955, P41=0.6908449421137701, P42=0.7506832740248234, P43=0.7592207446622945, P44=0.7118445166174516, P45=0.715140114101132, P46=0.7046026824441265, P47=0.7034730168638414, P48=0.6984305181289878, P49=0.7372960726655949, P50=0.7210261311762128, P51=0.6871636689319686, P52=0.015938341851548674, P53=0.000956359059454766, P54=6.744054994745128e-05, P55=9.852660531909086e-05, P56=0.00020961839519461692, P57=0.001801359693668876, P58=0.0069353423269515975, P59=0.017228848718449104, P60=0.03048471136064918, P61=0.04866000931312205, P62=0.08143049328893798, P63=0.06645460658368883, P64=0.017010599418118513, P65=0.012590244240433457, P66=0.017235234054308216, P67=0.016514836333655938, P68=0.016605582848268883, P69=0.0290323570962597, P70=0.06316096341839528, P71=0.03092114071819235, P72=0.014980068932793196, P73=0.03261677254592519, P74=0.029050225725087517, P75=0.021695753835049975, P76=0.014194565304724793, P77=0.01457781651454335, P78=0.029782561306292282, P79=0.05832096495404993, P80=0.07420886123979899, P81=0.02712466796410093, P82=0.0106889357404005, P83=0.028060061640639778, P84=0.056434353254034494, P85=0.04905829239171514, P86=0.04690526153633231, P87=0.04028963312031604, P88=0.06102818644424303, P89=0.04309017825526941, P90=0.012796841090863036, P91=0.021468615843198043, P92=0.12182280810763212, P93=0.14298326410169854, P94=0.02112415610782796, P95=0.006886920692680764, P96=0.0066182673364791, P97=0.006004437997873434, P98=0.027758024875981453, P99=0.10881680818535339, P100=0.0821066160264748, P101=0.011216630672909112, P102=0.006144332200681629, P103=0.002512633547424776, P104=0.010819364125060545, P105=0.029118128802469386, P106=0.011007002787592372, P107=0.0040853638199680985, P108=0.00528356224409654, P109=0.008979558784948227, P110=0.0035937995907363114, P111=0.0010371581311743978, P112=0.0014557143876051148, P113=0.0009356290528787586, P114=0.0014390537775146853, P115=0.0005988717198000733, P116=0.0006736439391725302, P117=0.0007096196707191063, P118=0.0004315550602526775, P119=0.00031251046576153297, P120=0.00037654166676020264, P121=0.0004059724738161546, P122=0.00034756464478499164, P123=0.0002617792526116322, P124=0.00018332853423832355, P125=0.0003078436460551805, P126=0.00036406001773185297, P127=0.0002174898080494828, P128=0.00018709084232130429, P129=0.00017889913503206804, P130=0.00024336935216733735, P131=0.0003627115385697685, P132=0.00023677561566762704, P133=0.00016416168981738877, P134=0.00019510892265715862, P135=0.00013050123638969576, P136=9.116789897824172e-05, P137=0.0001061354734647592, P138=0.00017133245998066088, P139=0.00020977625819781532, P140=0.00019404482715143964, P141=0.00013111472776766248, P142=0.0002334173227787944, P143=0.0002945335513461917, P144=0.0004590700952435896, P145=0.00034909436191896863, P146=0.00032872824343788083, P147=0.00015668119275612734, P148=0.00010668185298481848, P149=7.894762550207329e-05, P150=8.483456938851838e-05, P151=8.297694033880949e-05, P152=8.816242099514322e-05, P153=9.452726114301407e-05, P154=7.878516501061514e-05, P155=7.277870169041972e-05, P156=8.34680317732752e-05, P157=7.799287838969443e-05, P158=7.928877643392764e-05, P159=0.00012777140797807788, P160=0.00016778632327334948, P161=0.0002031639354690266, P162=0.0002221905317987976, P163=0.00020472260442932363, P164=0.0001559983474098684, P165=0.00012109649322129492, P166=0.00011230863492018164, P167=7.575738602280854e-05, P168=0.00020144247335170066, P169=0.00011662447457847522, P170=7.621004444106195e-05, P171=4.436650032263429e-05, P172=1.2738065575087262e-05, P173=1.7160357344937408e-05, P174=1.3465685388222555e-05, P175=1.448834569968051e-05, P176=2.2592625914035123e-05, P177=1.718347354810004e-05, P178=9.234196542581765e-06, P179=1.7865518813734592e-06),Row( P0=-690.8758347634954, P1=83.23908751123001, P2=13.497107894515816, P3=20.717336450140344, P4=4.719233152147522, P5=3.1580302195108447, P6=4.133932819790201, P7=-0.22611315066542204, P8=7.56353838868441, P9=0.9689357265346887, P10=-0.7355097351630869, P11=1.0783667494766638, P12=-2.3252181421912157, P13=0.2958295976188826, P14=1.2602504363007851, P15=-2.9360503123259765, P16=-0.2883303579350059, P17=-1.6607144161353449, P18=-0.15084553602954945, P19=0.14735709500957547, P20=-1.6792823968683575, P21=-1.2044907782806733, P22=-2.2706472027755567, P23=-1.4026945948753975, P24=-5.4268633331271605, P25=-0.6399574146696043, P26=-2.617040870447802, P27=0.4999891719495994, P28=-3.024142894618035, P29=-0.06438516796620555, P30=-1.4679560403570249, P31=3.5122980559148687, P32=1.34930195936286, P33=2.99452254385648, P34=1.0767005641201486, P35=3.3339744539783123, P36=3.03318330201664, P37=5.334328928786001, P38=3.6664208489303527, P39=3.5099569105173956, P40=0.7591550704645138, P41=0.7496226575026185, P42=0.7102318745147034, P43=0.6767027716211717, P44=0.6733638512475445, P45=0.6537573505042221, P46=0.6501186207139431, P47=0.6502195630979225, P48=0.6651939019240722, P49=0.7634014060415323, P50=0.7703410602165826, P51=0.7701748676948794, P52=0.00011596410359240656, P53=4.7340028239702884e-05, P54=3.70964955614622e-05, P55=6.125587504878801e-05, P56=0.00024445979492869794, P57=0.00585958795539852, P58=0.029927307051263768, P59=0.05029832088334636, P60=0.023237292805109746, P61=0.002957666165512193, P62=0.0011391665651337615, P63=0.0010850677209044864, P64=0.004221073777057933, P65=0.019244877895858426, P66=0.026274380863217504, P67=0.023172372271408993, P68=0.021001811339806286, P69=0.010111906346802804, P70=0.002005817809308547, P71=0.0024702218441898945, P72=0.004744678898471702, P73=0.0061795778300493915, P74=0.009570226296883502, P75=0.008718583204658525, P76=0.009124459740872184, P77=0.024417145048157233, P78=0.018735831170830785, P79=0.016142628853600835, P80=0.006123750542543001, P81=0.003021339028894179, P82=0.002925778573455984, P83=0.0010207038130251372, P84=0.0015077933189313612, P85=0.004151667763183329, P86=0.004049311792418963, P87=0.0023887267669880138, P88=0.010702032683910078, P89=0.008362341352477537, P90=0.005126433729297117, P91=0.016334883798185408, P92=0.005746778087507243, P93=0.005028581919197868, P94=0.027086309341229634, P95=0.012454527849813449, P96=0.004775329763918184, P97=0.0002621975433428931, P98=0.0018710302931659257, P99=0.0035623297679782495, P100=0.0009245592746484987, P101=0.00025992402547566056, P102=0.0012701734867798845, P103=0.006645958388318059, P104=0.0009565485799733415, P105=0.00019497132014967438, P106=0.0029158580302394554, P107=0.0043386642708755935, P108=0.00048142858644162405, P109=0.001680376917877297, P110=0.0033459717173752743, P111=0.00036165462937912953, P112=0.0007062218171016065, P113=0.00036527706268179775, P114=9.388415928442431e-05, P115=0.0003620059721204169, P116=0.00012353613993460431, P117=7.91228969618969e-05, P118=6.598397260231266e-05, P119=5.813605402586807e-05, P120=8.74977391169294e-05, P121=0.000123043726133528, P122=0.0001141285117342778, P123=2.7742138399442934e-05, P124=5.130313353674929e-05, P125=2.464327731987951e-05, P126=3.87137068355654e-05, P127=2.9798226922360583e-05, P128=2.6788979014786952e-05, P129=3.4162420821708415e-05, P130=4.0033815396567453e-05, P131=3.109579577441035e-05, P132=2.92836254610894e-05, P133=5.0443259465314236e-05, P134=3.240096509449551e-05, P135=2.606872790366781e-05, P136=4.1245227788280096e-05, P137=3.710655531324327e-05, P138=2.870897034682571e-05, P139=3.054591788529211e-05, P140=2.823021625171341e-05, P141=3.473034528792915e-05, P142=6.388300278179827e-05, P143=8.158595414668798e-05, P144=3.369635154218558e-05, P145=2.9152846835206194e-05, P146=1.6998762669090566e-05, P147=1.3834611548070529e-05, P148=1.0679038750309763e-05, P149=1.201047985178366e-05, P150=9.680786872377012e-06, P151=2.126642639417849e-05, P152=2.4154154218346917e-05, P153=2.1181187110095778e-05, P154=8.005471959486348e-06, P155=1.313670034597278e-05, P156=1.831484290051208e-05, P157=1.5848545607405398e-05, P158=1.1729873785105829e-05, P159=1.466139464003547e-05, P160=1.132227474138087e-05, P161=4.466190547189371e-06, P162=2.6709301771600835e-06, P163=3.0208942326532284e-06, P164=2.027603583919919e-06, P165=1.2756427485079847e-06, P166=1.4590685265741267e-06, P167=2.1089650997830178e-06, P168=1.3920659537079824e-06, P169=9.474074989254013e-07, P170=1.3018838922619771e-06, P171=1.1678377056653547e-06, P172=6.530890851069109e-07, P173=9.380468473919875e-07, P174=1.4510075286376565e-06, P175=2.6483969385167935e-06, P176=2.6795362699677378e-06, P177=2.2260915633462545e-06, P178=9.794472171588724e-07, P179=1.0629191885807942e-07),Row( P0=-460.0148375745035, P1=30.214365224167484, P2=-7.8139001847924385, P3=7.418289695521701, P4=-2.271393083703019, P5=-0.5660032640170416, P6=-1.922473177806749, P7=1.430634066022678, P8=3.6078830483846054, P9=2.554940840001979, P10=-2.909658561539149, P11=5.934793854187921, P12=4.2557301637245875, P13=6.695478766594503, P14=13.619848025379127, P15=10.188177420276874, P16=10.54275199013786, P17=8.372994743091203, P18=3.602237467528862, P19=10.226388219585674, P20=5.014099048747365, P21=8.675576927226276, P22=5.101908191526471, P23=2.9395728703276163, P24=4.289526085352952, P25=6.352970368869238, P26=5.973444942016429, P27=5.254704270150004, P28=1.8045183024775104, P29=2.8467621579069604, P30=0.7425270913903914, P31=3.493995804322717, P32=1.4877217703242467, P33=1.2930152981597764, P34=0.6378928485808729, P35=0.48507293508184146, P36=-0.04146081838317471, P37=2.7901242563268016, P38=2.1103254048458115, P39=1.9151685882674343, P40=0.7033859470277148, P41=0.7137683989590377, P42=0.7255705686202122, P43=0.7460144243770325, P44=0.8061476471316428, P45=0.7912587851918624, P46=0.739209393450644, P47=0.6931492438585238, P48=0.7150612867609292, P49=0.7445617255861584, P50=0.712767894463789, P51=0.718253208105799, P52=7.242942668870797, P53=1.3832427100835003, P54=0.1343033596920329, P55=0.016681113524306394, P56=0.006236751621278664, P57=0.0028856050851343656, P58=0.0014314815850596772, P59=0.0036846535928403367, P60=0.010616831610533747, P61=0.05127692499100471, P62=0.25691479829188274, P63=0.3876954012353907, P64=0.2724634765619741, P65=0.48593102412588934, P66=0.5699623568808544, P67=0.4985301289094488, P68=0.6952408105448632, P69=1.1986532172885689, P70=1.9773877399923014, P71=3.573509685052147, P72=3.5709049077841524, P73=1.8536712849914794, P74=0.303788325770064, P75=0.03351341617304791, P76=0.02603486014038104, P77=0.05866311817449582, P78=0.14195832877183126, P79=0.33434716294631944, P80=0.2836318596906943, P81=0.10425184744003642, P82=0.13651245763410838, P83=0.26299151245818525, P84=0.4367112830924459, P85=0.41381769261168055, P86=0.38066695439875353, P87=0.4989101971968111, P88=1.0030622963723657, P89=1.0216427856042969, P90=2.3547934912887927, P91=6.9145016162574375, P92=4.846838023412705, P93=1.4306129863520944, P94=0.1840106346990086, P95=0.08219603302345639, P96=0.07187530814410546, P97=0.11487859260503162, P98=0.13621745995098228, P99=0.18267645126656695, P100=0.4845587674017064, P101=0.8203702115346904, P102=0.7729950940365097, P103=3.511438277041586, P104=2.40794138309099, P105=0.1853144281798941, P106=0.0959806795415337, P107=0.1624489249425246, P108=0.3537658326678468, P109=0.10164369125085812, P110=0.16500141194222268, P111=0.7684699598182068, P112=1.6983228079500987, P113=0.6655807783341272, P114=0.18598491566165679, P115=0.029157104071949536, P116=0.042519040756202296, P117=0.29289698768211103, P118=0.3584436079128105, P119=0.13787332024242208, P120=0.04016807578177049, P121=0.06429990721668641, P122=0.053915358059477794, P123=0.027041400961945922, P124=0.03187259313091296, P125=0.03204684409031323, P126=0.01682521042445269, P127=0.005677261627186568, P128=0.011682387296030773, P129=0.025646574282423378, P130=0.01073926195164231, P131=0.016508425423899763, P132=0.031294378684959386, P133=0.028614685152898976, P134=0.020059021337850442, P135=0.017452502977772674, P136=0.011835523784642291, P137=0.010759726951918173, P138=0.00689061122436723, P139=0.008665357795924299, P140=0.007361040934028272, P141=0.015719738215569407, P142=0.015993158782061014, P143=0.030835522207750363, P144=0.02248056203539054, P145=0.02045528255744432, P146=0.01422487579871502, P147=0.009061686653343632, P148=0.010042303866302511, P149=0.006714193318841723, P150=0.004730977605924177, P151=0.002819411343619408, P152=0.0018592721031359665, P153=0.0023512846505523385, P154=0.002778327425958924, P155=0.0027347965519439565, P156=0.003685995026500319, P157=0.004349649321425007, P158=0.004114123799083934, P159=0.0053040378692049105, P160=0.007186518195546047, P161=0.007634329069607473, P162=0.006487402779875046, P163=0.004840935539624875, P164=0.00485265620164241, P165=0.0031233253213436046, P166=0.0028009950276985346, P167=0.003069056796091375, P168=0.0053453876148364, P169=0.005887544420061397, P170=0.004667402193548011, P171=0.008356575292159834, P172=0.008461974481128034, P173=0.007478767719706417, P174=0.0054753411128847206, P175=0.005624704661609266, P176=0.00366625318223496, P177=0.002751099707232779, P178=0.001251143774580851, P179=0.00014354892945530294),Row( P0=-572.3992007697968, P1=64.04950306156552, P2=-11.833391873684368, P3=8.63319007845382, P4=-8.385476297642995, P5=-6.893464771571535, P6=-8.836018804138671, P7=0.8230679300805517, P8=10.459796523466629, P9=-5.561680526706486, P10=-5.8128917326329, P11=3.3089040563504417, P12=-1.985492208741545, P13=-2.942538119714519, P14=3.595909706345041, P15=-3.970629382239743, P16=-1.1361129013085989, P17=1.1605770677245808, P18=-2.6020994893556386, P19=1.8038843532336521, P20=1.7596701518124611, P21=4.738580101239566, P22=1.415324867736014, P23=4.604459769986651, P24=1.8729212815569956, P25=6.3613546339130975, P26=6.581247729674678, P27=8.36138054777402, P28=3.299300375669942, P29=3.7592382993274414, P30=0.7433629595167941, P31=5.615941384064306, P32=1.1899198317650468, P33=3.3294506602493974, P34=-0.7873324083753505, P35=0.4688249907810711, P36=0.09912498790297784, P37=1.73789475559643, P38=-0.9310687488587066, P39=1.1215098233213125, P40=0.6305447792521577, P41=0.6590684897295592, P42=0.6722854928172862, P43=0.7202992040185329, P44=0.7140336802025553, P45=0.7064038333213631, P46=0.7176672432740636, P47=0.6934505310114693, P48=0.6776497197081781, P49=0.7141340458922597, P50=0.6933165856165837, P51=0.6265557297488208, P52=0.0006797392908727882, P53=0.0001206634734007214, P54=4.097035577728692e-05, P55=5.823515717302196e-05, P56=0.0007063730022995639, P57=0.005332435951787143, P58=0.010207497372952694, P59=0.027432293055695587, P60=0.1447405085792678, P61=0.33954947178292044, P62=0.41088582745644736, P63=0.2702676486938883, P64=0.1198129822936634, P65=0.038454081788808867, P66=0.00895391664738734, P67=0.01059900168986005, P68=0.015051884963781018, P69=0.03872526762116826, P70=0.08881518165501741, P71=0.06470793690323737, P72=0.043257062647418514, P73=0.06692650602890676, P74=0.08436161538705991, P75=0.05949241305176743, P76=0.07722782445875861, P77=0.35382186039777636, P78=0.290304111933812, P79=0.09697141204674443, P80=0.058504928344841466, P81=0.04389493271409449, P82=0.05658783987330439, P83=0.13248666484251742, P84=0.22071630946077922, P85=0.18253079513713097, P86=0.08694963042005643, P87=0.06545398945408883, P88=0.11787649590516577, P89=0.14173522090450766, P90=0.15322014797842062, P91=0.08512350118194278, P92=0.06389710858844573, P93=0.03527751971810537, P94=0.018405274022696365, P95=0.012476894626962982, P96=0.018650142342107972, P97=0.024985549773665145, P98=0.028417406908223315, P99=0.022651082207425944, P100=0.010072577058532404, P101=0.009845663603748025, P102=0.006835191175594339, P103=0.005785774482341814, P104=0.005046666898833219, P105=0.006514608397270608, P106=0.006926507211250812, P107=0.009064233508165338, P108=0.010966601448752352, P109=0.008478353406342206, P110=0.008438115651954849, P111=0.00587711063121271, P112=0.0038802950435007956, P113=0.0024052729847525867, P114=0.0014854789357509708, P115=0.0019393839488526929, P116=0.003939222469036891, P117=0.002422184774524879, P118=0.001359780646565581, P119=0.0020986404060129344, P120=0.0017777981245382084, P121=0.0012519098134194735, P122=0.0014674767293691709, P123=0.0008065794314226265, P124=0.0003271760853182993, P125=0.000387217761601331, P126=0.0005217747206602801, P127=0.00029608874319770484, P128=0.00030962816572672184, P129=0.0002116441303472387, P130=0.00024383861573640494, P131=0.00020866145578134231, P132=0.0001639633906813519, P133=0.00020834048546136653, P134=0.00011573714024118496, P135=6.52026737412412e-05, P136=9.434112563638856e-05, P137=0.00011017654459264544, P138=0.0001333110664388394, P139=0.00018999847783090834, P140=0.0004244384865511116, P141=0.0005871697756893271, P142=0.0006788431690294869, P143=0.0006348356868619598, P144=0.0005437954246410322, P145=0.0004375109398113294, P146=0.0004817581917204824, P147=0.0003925102983090504, P148=0.000249418522088605, P149=0.0006317722351355711, P150=0.0005216485658042949, P151=0.00037990163456276803, P152=0.0004656087491515654, P153=0.0004212426777120469, P154=0.0003489345269606257, P155=0.0004162929563150816, P156=0.0002732556623589182, P157=0.00030392515476608886, P158=0.00036558212135484646, P159=0.00045688719316231324, P160=0.0004637604818902305, P161=0.000394339152652558, P162=0.00044478122840885785, P163=0.00035197892811779244, P164=0.00015527994404638596, P165=0.00010878264625460467, P166=0.0001618174747221851, P167=0.00018675305455232816, P168=0.0001578079861560054, P169=6.705081992105374e-05, P170=2.659766339954183e-05, P171=9.156488769446777e-06, P172=1.4857869412720165e-05, P173=1.614675105429833e-05, P174=2.2172738311125535e-05, P175=2.6488291132844918e-05, P176=4.0216878356943666e-05, P177=3.892634230064713e-05, P178=2.1849296830928388e-05, P179=2.4194771406556873e-06),Row( P0=-508.01716017097783, P1=34.293856500005475, P2=-13.48471074305677, P3=0.6861148623915732, P4=-7.6082904588647295, P5=-5.998632436527938, P6=-11.117157393797845, P7=-0.747899044601937, P8=-1.333285189626302, P9=-6.47954671965492, P10=-6.714151004533967, P11=3.1280988106696395, P12=5.074924038452893, P13=6.2644512539506065, P14=13.586737676849067, P15=7.367703510014967, P16=8.115365297463653, P17=5.752200592882748, P18=4.442063251685675, P19=10.191507464844918, P20=2.2856617334311777, P21=7.026183434011884, P22=1.76524460768371, P23=2.044519468576791, P24=3.6639897961302723, P25=4.028865544524695, P26=2.2985871979095442, P27=1.383086347379412, P28=-0.9732401842069072, P29=1.4704311195630966, P30=-0.3749517854571048, P31=2.5622523705881073, P32=-0.10323131666595366, P33=2.5351562734128508, P34=1.3663025530878654, P35=1.2348815959863289, P36=2.461424513899597, P37=2.4961089355916677, P38=0.3861579992124373, P39=-0.16592778052950702, P40=0.664948904698843, P41=0.6069744219140474, P42=0.6431737311211039, P43=0.6924218531787505, P44=0.6571342510220216, P45=0.6985586233537525, P46=0.7206747920267668, P47=0.6736666683851796, P48=0.6154797778870099, P49=0.642128902136073, P50=0.6898678409064082, P51=0.6832107382081963, P52=0.04767319449044829, P53=0.0021726239948540852, P54=0.0007186039246058661, P55=0.00043188349509505885, P56=0.0002984428799155695, P57=0.0004035683275834234, P58=0.0009595234998787672, P59=0.0014234024423674726, P60=0.0016548333668530216, P61=0.009851321963356929, P62=0.08435949291697506, P63=0.2721312678621765, P64=0.2640515422883123, P65=0.42777589299059793, P66=0.6290840289086188, P67=0.5584101140415854, P68=0.5192243001248528, P69=0.508948761038142, P70=0.5761059582051167, P71=0.7939102278398217, P72=1.8616456460992732, P73=2.5110748918169206, P74=1.6330282023493616, P75=0.5022017617976771, P76=0.06549174915994918, P77=0.03961488619984152, P78=0.04441572916507994, P79=0.06607553486529065, P80=0.07665398159542286, P81=0.10973584762834372, P82=0.13499757131429815, P83=0.13875819894160157, P84=0.17244349899234435, P85=0.20735322455326735, P86=0.1652757158616236, P87=0.23053176110970558, P88=0.6420670386665793, P89=0.5151053393039491, P90=0.3427417668120322, P91=0.3763278113263003, P92=0.5112838536526495, P93=0.41078057666890944, P94=0.6316994622167231, P95=0.514612524370008, P96=0.052842931019926376, P97=0.05873269283331027, P98=0.05419006908261581, P99=0.03190104333599837, P100=0.05924935927995974, P101=0.0628881326448801, P102=0.02777379025900011, P103=0.02753704320540194, P104=0.02084906397781507, P105=0.01917329705232108, P106=0.022367054728700486, P107=0.057142830773549175, P108=0.0460549940089449, P109=0.030282732949490906, P110=0.028211041375429825, P111=0.02993621042419397, P112=0.04245703188470908, P113=0.044177694722521546, P114=0.02755761268934475, P115=0.02905518352727823, P116=0.0172325706784276, P117=0.01092876119671919, P118=0.010647390341536181, P119=0.009797089973136018, P120=0.007753131906392353, P121=0.00858393920093786, P122=0.005740323919156137, P123=0.0030897581134711706, P124=0.0033579362720367136, P125=0.011740148057827776, P126=0.016265196150182588, P127=0.003986069020814451, P128=0.0012001344350215527, P129=0.001538287090658994, P130=0.0055064410667829055, P131=0.0030179544822106453, P132=0.0012157758867480213, P133=0.004587870722802438, P134=0.004611673809345278, P135=0.005342621871848325, P136=0.0022951472415023264, P137=0.0007178277328391899, P138=0.0006512417111296508, P139=0.0004779049321947035, P140=0.0009821377678037712, P141=0.0011858220836527197, P142=0.001757690415801241, P143=0.0016786246545685286, P144=0.0025864531343494624, P145=0.0031682655812281728, P146=0.0026552845654974344, P147=0.002296041053840982, P148=0.001744669476657065, P149=0.0018067063364422362, P150=0.0026648479131426927, P151=0.0026890318704388, P152=0.00234691734438122, P153=0.0019555188784139243, P154=0.002366576652250518, P155=0.002052185125127306, P156=0.0028459069934567843, P157=0.003153830592453294, P158=0.002948365851462473, P159=0.00348855553410805, P160=0.002951849628590199, P161=0.0025291715800893682, P162=0.0020420247739035956, P163=0.002291197927200889, P164=0.0015991672776005513, P165=0.0017489380703095757, P166=0.0013866250750457956, P167=0.0010511176041656486, P168=0.0008538529000026887, P169=0.000827551037342994, P170=0.0008156377521917346, P171=0.0006584952413221605, P172=0.00029007261771032923, P173=0.0003769788651037029, P174=0.0004041673179466998, P175=0.00030728182662270346, P176=0.00028594042319861606, P177=0.00031214155424913773, P178=0.0001836922473332956, P179=2.1195938895401695e-05),Row( P0=-554.2004234483849, P1=35.06748041114167, P2=-0.1423125050349442, P3=5.302238794905276, P4=-2.5997407088668916, P5=-2.257880284467285, P6=-5.974136060138005, P7=6.958042600329903, P8=3.3271881509845094, P9=-0.670393230879847, P10=-6.783302312574687, P11=7.31983097751865, P12=-1.7282039977406622, P13=2.6476669104355186, P14=5.785581096807953, P15=2.2451240212001338, P16=3.514407639322378, P17=4.341611510631111, P18=2.32580145515124, P19=4.0116007767470965, P20=3.6260297745042562, P21=2.1660233988091453, P22=2.736850541235635, P23=1.105165498803023, P24=-0.8804782379768054, P25=5.913686495678926, P26=-0.8643962623051997, P27=3.2533919600662387, P28=0.367066319589804, P29=-0.518209189298328, P30=0.015435043407703164, P31=1.8308048331172924, P32=0.8520075716926249, P33=1.973382927061211, P34=-1.9803781687332462, P35=1.0201930450460022, P36=1.0764855502332695, P37=3.3759422991070145, P38=0.6044240071895475, P39=0.4005886714459719, P40=0.7210813684972787, P41=0.6491359199242637, P42=0.6450047820138265, P43=0.6593694818220243, P44=0.6433422329031298, P45=0.6666406719663647, P46=0.6650060741301462, P47=0.6436645223152161, P48=0.6773399860437473, P49=0.7745256172482097, P50=0.7865404499509135, P51=0.7591157516593998, P52=0.05022353495829601, P53=0.0023641904589414924, P54=0.00020422870942107805, P55=0.00032023052532919205, P56=0.00034138209098830537, P57=0.0013112395387529208, P58=0.003851527227550678, P59=0.011865347747040312, P60=0.020445644006205427, P61=0.017873452769995427, P62=0.03499077938106431, P63=0.07871865162207307, P64=0.07425155509861733, P65=0.07197612576412363, P66=0.11424179866927807, P67=0.08280843247129928, P68=0.04028669815095964, P69=0.06431199434740069, P70=0.14063085006695822, P71=0.24057890226308823, P72=0.4218099098332997, P73=0.7372165820045873, P74=0.3494311793386301, P75=0.01431954611318178, P76=0.010097226324328665, P77=0.00821567279895646, P78=0.008480409373632675, P79=0.009183947748111418, P80=0.010364598797558104, P81=0.02750224807827546, P82=0.056929765784749425, P83=0.05542081916435591, P84=0.09270589218272253, P85=0.09455607192467377, P86=0.08075064130230873, P87=0.06614140195558495, P88=0.0963540678900423, P89=0.04808759617577444, P90=0.03363490390411407, P91=0.018036511794959197, P92=0.033455998656121526, P93=0.0958927172751726, P94=0.040765112259377755, P95=0.0058927877428091545, P96=0.004450129304221148, P97=0.0040791306035023985, P98=0.00577501907319026, P99=0.010311492593410822, P100=0.010470631065724099, P101=0.007450303189972179, P102=0.003935937664085367, P103=0.0025527456172708585, P104=0.0019312102053272758, P105=0.0030166202113935925, P106=0.002514231464553443, P107=0.001014204030916013, P108=0.0012980224220525602, P109=0.0019334831019308927, P110=0.001003570316986893, P111=0.000940955824885082, P112=0.00390806427225623, P113=0.00884662533049242, P114=0.0036211789297585775, P115=0.002930956693840362, P116=0.001130002783768731, P117=0.0015118079198815754, P118=0.00138379159715466, P119=0.0009477027025829919, P120=0.001121959495066474, P121=0.0014362206915054037, P122=0.0009993527172216616, P123=0.0010257085639837724, P124=0.0013317019659710289, P125=0.0013524467559650746, P126=0.0005457347634233424, P127=0.0002449708740988713, P128=0.00011879463350704446, P129=0.00012074610953732454, P130=0.0001854643635163676, P131=0.00020255149773362583, P132=0.00014550693249404202, P133=0.0001719567707092061, P134=0.00016462818310194365, P135=0.00012468822057462743, P136=0.00016567201254363687, P137=7.899578549215287e-05, P138=8.221266519635825e-05, P139=0.00015012998648649892, P140=0.00021228929664500318, P141=0.0003473105852456436, P142=0.0006495565055285166, P143=0.0007947470638846367, P144=0.0013837144161040317, P145=0.0010343608371720348, P146=0.0007455290512626054, P147=0.0005740987976789508, P148=0.0004180093451128521, P149=0.0005317703575160825, P150=0.00038728852387874936, P151=0.0006595518917649548, P152=0.0006935838903380619, P153=0.00069024043077734, P154=0.0006592296994263759, P155=0.000573411979847543, P156=0.00045238083122738104, P157=0.00036555090075225346, P158=0.0004246572963415867, P159=0.0006199675708194196, P160=0.0011477404597381039, P161=0.002122021276828384, P162=0.002793741041953157, P163=0.003677514775985288, P164=0.002226135645075512, P165=0.002390697813749241, P166=0.002633976049790697, P167=0.002813352868512473, P168=0.0029949271608206538, P169=0.0034538666632128964, P170=0.002666541184608937, P171=0.0010385502783473363, P172=0.0003258201433496183, P173=0.00028156835731669766, P174=0.00035594559626292454, P175=0.0002030971535855435, P176=0.00017228442773361446, P177=0.00018300587168953522, P178=9.403990303193413e-05, P179=1.0911795964250209e-05),Row( P0=-609.3404119852243, P1=61.82565550803376, P2=6.929041259420285, P3=11.81266353356595, P4=0.27376844355011326, P5=-0.727565984757579, P6=1.2404914608020954, P7=1.7896485229219443, P8=3.8455727744349795, P9=3.07712550247192, P10=-1.9608354684484894, P11=2.496612953078978, P12=-1.6781354874883072, P13=0.14579479424765437, P14=2.997602865541674, P15=-0.22381290340227303, P16=0.6901315070312941, P17=2.796330348116579, P18=2.2731900595659194, P19=2.3487257666792316, P20=2.215163791033853, P21=2.25333053864366, P22=2.3515355861272145, P23=3.4640186791104917, P24=2.158577175343956, P25=4.433072424976563, P26=3.2995117453778073, P27=3.2947832133978614, P28=1.6559089449371096, P29=0.6393006138841705, P30=-0.5829967675237425, P31=1.3149641264533627, P32=-0.3605069854026496, P33=1.03953842180861, P34=-0.9285208930227997, P35=0.6578200149777562, P36=-0.005702051439120748, P37=1.3540819497701952, P38=0.2906752166371306, P39=1.4921047778788143, P40=0.7401581424855291, P41=0.7297825577302232, P42=0.7667299612792636, P43=0.7526949125308612, P44=0.7554807236578247, P45=0.7245122003272734, P46=0.7205609691034897, P47=0.6828030524775215, P48=0.7214235829394375, P49=0.7518810749401144, P50=0.7520830165093407, P51=0.7661835410437865, P52=0.4648065875450563, P53=0.025730931455835125, P54=0.0010152955403539884, P55=0.0003407167794819288, P56=0.0004312705883559035, P57=0.0028925834630764485, P58=0.007560071292325438, P59=0.014238304013758415, P60=0.03098377984761238, P61=0.0876463063108133, P62=0.10587855458235271, P63=0.0721396856475334, P64=0.053258048411625035, P65=0.04239889675219768, P66=0.023858137404962988, P67=0.011337645246236271, P68=0.006658113983561742, P69=0.013684876416694308, P70=0.031382325168246905, P71=0.03882335409867689, P72=0.04768746559726526, P73=0.03832923993743337, P74=0.03241283289442058, P75=0.030028032792473867, P76=0.024417914617919714, P77=0.019810110882665863, P78=0.030446278918856574, P79=0.038842048093718315, P80=0.027715710724270835, P81=0.03064992744968123, P82=0.05653808666357475, P83=0.06644071529204007, P84=0.059301578792961995, P85=0.03570296104035803, P86=0.03422750947037875, P87=0.021687292636838454, P88=0.017980993813561143, P89=0.012993921995419844, P90=0.04077108189934583, P91=0.18443631215607378, P92=0.10520470875072582, P93=0.0044009353974339025, P94=0.00282635001122799, P95=0.00520082650707372, P96=0.006219573093918899, P97=0.029018362975149963, P98=0.06170978075380268, P99=0.03083953463755601, P100=0.004108567665855973, P101=0.002933495054118979, P102=0.003517229402091278, P103=0.02617151295712113, P104=0.041508763938807675, P105=0.007032912621204196, P106=0.003114672514365051, P107=0.003427414615448371, P108=0.01421451602112996, P109=0.03494305817864316, P110=0.006705045016876276, P111=0.0010271536557852007, P112=0.0023151738818541706, P113=0.0028326411190352013, P114=0.001486426119074592, P115=0.0007244035313720029, P116=0.0010694395018554443, P117=0.0016005046889368125, P118=0.0002940292921763827, P119=0.0003128944604083, P120=0.0002910110177981731, P121=0.00029055841428709216, P122=0.00036591954006856174, P123=0.00021501131788385528, P124=0.00019527184672301124, P125=0.0002781985513839917, P126=0.00023259914481593927, P127=0.00023862016329359025, P128=0.0002773762598148462, P129=0.0003955863849003759, P130=0.0002495933893977095, P131=9.850697725619832e-05, P132=7.256815886369026e-05, P133=7.283029757846864e-05, P134=8.7903561782498e-05, P135=0.00014525393710169293, P136=0.0001630864386059721, P137=0.0002522144098595646, P138=0.0001762959297938844, P139=0.00021517760939333166, P140=0.000274781392960496, P141=0.00021018383451572358, P142=0.00031716062887983247, P143=0.00021513931636775322, P144=0.0001965806871332458, P145=0.00015762860293897388, P146=0.00015838319470794392, P147=0.00015934035888004398, P148=9.893645604870075e-05, P149=9.379212678519883e-05, P150=0.00012983181882683393, P151=8.903285028447193e-05, P152=8.112438630609707e-05, P153=0.00010099735224368201, P154=9.075456532356156e-05, P155=5.60267755912231e-05, P156=6.838335055581247e-05, P157=0.00013760879439230246, P158=0.00010986187847971971, P159=0.00010161326761286011, P160=0.00021652962404777035, P161=0.0003061993975983563, P162=0.00036569427705001494, P163=0.0002650805944664922, P164=0.00010452593447785316, P165=0.0001315312937194382, P166=6.95221274988935e-05, P167=8.985781863171942e-05, P168=0.00010060891052448893, P169=0.00016248257255592697, P170=8.17151257477453e-05, P171=1.3806480817673587e-05, P172=1.2258929149058817e-05, P173=1.4673623511705664e-05, P174=1.858730115719553e-05, P175=1.1892831588488197e-05, P176=1.1293137311944882e-05, P177=1.2651418986649577e-05, P178=5.653276412179869e-06, P179=5.051954713409886e-07),Row( P0=-562.0642618740728, P1=46.90658692426649, P2=-0.2464115785349937, P3=2.9255448686573233, P4=-3.6309826097815163, P5=-5.1841167974041715, P6=-4.516012914860169, P7=2.3201961635731956, P8=-2.4301951507199244, P9=-2.1982153384424734, P10=-4.5573480817217655, P11=4.443889257816871, P12=-2.349485492303356, P13=4.946070998332217, P14=6.121084225117273, P15=1.9245431204693502, P16=4.1165015917524, P17=2.561503267019975, P18=2.5701764690535325, P19=2.1629492939482464, P20=1.2857959069556504, P21=1.6085096856174594, P22=2.9289348481473363, P23=2.307126358461547, P24=0.724164256392547, P25=4.568430362521301, P26=0.41190459126442786, P27=2.6820149714548065, P28=0.8196274766873675, P29=2.009371492857248, P30=1.2168335650907498, P31=3.5136425729192795, P32=1.3974194255752106, P33=1.9007724687376848, P34=-0.10536499406059298, P35=-0.10871930715679289, P36=1.417451908215929, P37=2.1466397892421893, P38=-0.2364640657636318, P39=0.6191932716864694, P40=0.7051706320204497, P41=0.6936469504203994, P42=0.6980142248869975, P43=0.6559961566382556, P44=0.6438931117087693, P45=0.6585823787935481, P46=0.6596970905225357, P47=0.6022390489591989, P48=0.6354600952528711, P49=0.7219565853398954, P50=0.7276019933807066, P51=0.7039213684469372, P52=0.0024940967586055893, P53=0.00031071263723209043, P54=0.000246426655684023, P55=0.00035878239985174485, P56=0.0002860732734431202, P57=0.0009167374486837265, P58=0.005776527250240292, P59=0.01986025423242861, P60=0.026851672025753004, P61=0.023718297389927652, P62=0.041547373461475974, P63=0.05200454116525765, P64=0.03653393845590524, P65=0.07343837955384645, P66=0.10367733557621997, P67=0.08637804798343265, P68=0.07527555418089331, P69=0.22571410703308778, P70=0.48081385385523134, P71=0.31509703393082444, P72=0.3599804023312325, P73=0.8866735539504652, P74=0.569990328637049, P75=0.04176664390307324, P76=0.01027997646611159, P77=0.012981638850221187, P78=0.019573189592586897, P79=0.028147439795123925, P80=0.027052706966039765, P81=0.04409958774993959, P82=0.0831029732990611, P83=0.15180473073813863, P84=0.16184633215025077, P85=0.11048997477178867, P86=0.07965835920047819, P87=0.07389551638062565, P88=0.11060067455133041, P89=0.08161473749660572, P90=0.059306563762836814, P91=0.07055337259227952, P92=0.13905526142412913, P93=0.14987637432252565, P94=0.07454500803422842, P95=0.08514012402105348, P96=0.08604854600629912, P97=0.013131493937501793, P98=0.007836015557491423, P99=0.009582329293633658, P100=0.013908397259762199, P101=0.024450958513678508, P102=0.016836743480252127, P103=0.011077546741983135, P104=0.02928187843245981, P105=0.023542481026929185, P106=0.005803787205146661, P107=0.004126777751376689, P108=0.004051409725614642, P109=0.0056862733132850805, P110=0.0027124098144465487, P111=0.007718312399517725, P112=0.004193987125342135, P113=0.002579841957883946, P114=0.007000202911871507, P115=0.003625541522795088, P116=0.0006050427191044533, P117=0.0009300483407840577, P118=0.0005721005139005056, P119=0.0010243537955648134, P120=0.0015253958105648216, P121=0.0018186569620575997, P122=0.0018662462517398898, P123=0.000958253087132221, P124=0.0009420133397117817, P125=0.0010617948617887528, P126=0.0013914842084318331, P127=0.0012441084644104054, P128=0.0010889603639537904, P129=0.0012264750384794467, P130=0.0007528397315916244, P131=0.0006984382635077923, P132=0.0005425017694034564, P133=0.00040086543905620646, P134=0.0004643873266315639, P135=0.00038727213041925496, P136=0.00036090698652371724, P137=0.00014460497395741727, P138=0.00014206250767103586, P139=0.0002528759980588099, P140=0.0001899466190634928, P141=0.00021064327829152903, P142=0.0005816066498548182, P143=0.0007636464509425407, P144=0.0006693647603516349, P145=0.0006905109272435892, P146=0.0006295273170071041, P147=0.0005003279746502055, P148=0.00044517452887116713, P149=0.0007330317946046946, P150=0.0003942526288043937, P151=0.0003048652492788662, P152=0.00022836323630440428, P153=0.0002477702248862077, P154=0.0002936433897903599, P155=0.0002450454264184245, P156=0.0002782631476459819, P157=0.00021942039244307354, P158=0.000325730069817803, P159=0.0003243784660781316, P160=0.0003757928043759184, P161=0.0007368433645939114, P162=0.0009617920830020279, P163=0.0005960939003178279, P164=0.0007477644970165085, P165=0.0007879010046415683, P166=0.0011324719658630492, P167=0.000897847044085732, P168=0.000953037090375851, P169=0.0005414350907367089, P170=0.00027048010190069245, P171=0.00011188023932951497, P172=7.908565810431663e-05, P173=7.194122856719114e-05, P174=4.175377013646827e-05, P175=4.512496259499042e-05, P176=6.007683606488028e-05, P177=7.548467978657656e-05, P178=3.833816327278343e-05, P179=4.028564521732154e-06),Row( P0=-482.13600820740965, P1=30.86133908317985, P2=-10.599146560966924, P3=4.828765963299695, P4=-2.2544542995805044, P5=-3.5484562707513536, P6=-4.001447981268629, P7=1.471776086511241, P8=1.3186579621824774, P9=1.469704162274494, P10=-2.9516282914165033, P11=7.395160618570061, P12=8.468583593887844, P13=7.749813817040594, P14=13.322589856675812, P15=9.444840462520071, P16=10.035012984195648, P17=9.886146233943423, P18=5.5841675911840705, P19=10.495037394249499, P20=3.127244559803063, P21=5.289292189300151, P22=6.155706715785087, P23=6.044474985426758, P24=6.709259849227566, P25=5.659694018009092, P26=3.971306871027889, P27=3.945589404950614, P28=3.409901754624349, P29=4.033045550557204, P30=1.5610551570675264, P31=3.820717746445787, P32=2.3966451306378884, P33=3.8905318813254985, P34=1.0571615082364711, P35=2.659163971032915, P36=2.3350932543719947, P37=3.020705503170051, P38=1.9237576242161536, P39=0.6108388617741484, P40=0.6933609520130437, P41=0.6887421858271965, P42=0.7176395504623027, P43=0.7086410391816816, P44=0.7291261062634773, P45=0.7570410311789486, P46=0.7947655272261271, P47=0.7242047323846061, P48=0.7042555542075755, P49=0.7524553032332215, P50=0.6990325269750068, P51=0.7079825179450815, P52=0.9138643629458243, P53=0.07674160320635898, P54=0.002918344439114397, P55=0.00037782695203423855, P56=0.0004305323673147603, P57=0.0008717093672623757, P58=0.00112106217246903, P59=0.0028136016849039974, P60=0.013260859419878299, P61=0.040222646372597, P62=0.05140995686676327, P63=0.09866873981908669, P64=0.21164763566897052, P65=0.45285445310085026, P66=1.0353212183296565, P67=1.6062726474100912, P68=0.8373664093690538, P69=0.8184747719841519, P70=1.1837056280130893, P71=1.7641693807406127, P72=2.304460742327094, P73=3.6407072407770906, P74=4.114117052283353, P75=2.334364258598948, P76=1.4210337347629063, P77=0.3439183132742178, P78=0.0565124163470335, P79=0.06232639991303127, P80=0.08475419299857717, P81=0.2746382736304277, P82=0.7324795304929762, P83=1.061430469913737, P84=0.748733796279899, P85=0.5061732521544154, P86=0.5568855095030045, P87=0.7897974686878093, P88=1.4692291962974022, P89=1.0600071780074387, P90=1.0631727200826657, P91=1.1221074110917781, P92=1.244054499384548, P93=2.0636104049886494, P94=4.213834976907584, P95=2.462056424436486, P96=1.426194081175487, P97=0.7107227613182606, P98=0.22333326374325568, P99=0.25485112952883104, P100=0.34751173477563396, P101=0.280274614855693, P102=0.1765391864532458, P103=0.2599919982553873, P104=0.29552320746656296, P105=0.6017037828535524, P106=1.8798669548860567, P107=2.2374378096758316, P108=0.9541023120254575, P109=0.37691987852544073, P110=0.16959376630547385, P111=0.5642366694540271, P112=0.15043626640522148, P113=0.16459608614748386, P114=0.6224435087833832, P115=0.6625116891749624, P116=0.3602787170624214, P117=0.1824627097833711, P118=0.03287884810499365, P119=0.04533263914293433, P120=0.14949514984957119, P121=0.2696615410403077, P122=0.062186957529307973, P123=0.028367117718989532, P124=0.10233755451329456, P125=0.10152227908748802, P126=0.02688180825038967, P127=0.03477485259851801, P128=0.028331153820482746, P129=0.05190703768598946, P130=0.05480678060240317, P131=0.049139679879305016, P132=0.05033990329811551, P133=0.06152196395211046, P134=0.03873646479680445, P135=0.04519114603796597, P136=0.021115247694889632, P137=0.015284515618080035, P138=0.013516867178737063, P139=0.022397583359964057, P140=0.02085665118210817, P141=0.028664065026685163, P142=0.04040066221646317, P143=0.047343646865675985, P144=0.024654006300045012, P145=0.016028169135532556, P146=0.017356385588709843, P147=0.008566081619448976, P148=0.00399907363803845, P149=0.0035774490845479497, P150=0.003406559869493895, P151=0.002825425225928359, P152=0.002615827072254385, P153=0.0026501438483662785, P154=0.002777180662636537, P155=0.0031552258308146573, P156=0.006627717337079785, P157=0.010196306356297102, P158=0.013412451221473087, P159=0.011923852785830632, P160=0.011258669248741695, P161=0.010140804481031775, P162=0.006725734521519018, P163=0.004181928164406974, P164=0.0030672152712283986, P165=0.002577086176666093, P166=0.0029963665806335364, P167=0.003916591242420802, P168=0.0046162931195775836, P169=0.005760528441997916, P170=0.0037691606174625756, P171=0.0036517931641726537, P172=0.0032768712093539445, P173=0.002675333396268261, P174=0.0025570682192091643, P175=0.002759461553183339, P176=0.002406104745262998, P177=0.001982459129190241, P178=0.001069057897996079, P179=0.00012375004464638376),Row( P0=-653.7794539147026, P1=76.07644246360627, P2=12.816397596760572, P3=15.847222477081006, P4=2.7529029676528376, P5=1.360398473678091, P6=-0.5114318666598963, P7=-0.3205474237959468, P8=7.183510484745215, P9=0.3441752362274343, P10=-2.1445072326139925, P11=4.00224191713807, P12=-3.968665973915911, P13=1.080142784766569, P14=-1.0017300962212325, P15=-4.1321763642649, P16=-0.8722041453845795, P17=-3.108015803967596, P18=-0.7684598780763671, P19=-1.6343046337957665, P20=-2.621147474379433, P21=-2.120200751617718, P22=-3.7886373418943693, P23=-0.32615321235962447, P24=-4.865490347725445, P25=1.8078103969859436, P26=-0.591460350578571, P27=3.5304930553210663, P28=1.3352351122860184, P29=3.74670001906776, P30=1.4968504158594842, P31=4.003663573973824, P32=0.8041328985297312, P33=1.2472449863196189, P34=-2.2740550856415416, P35=0.9143481095604928, P36=0.02722436101645951, P37=0.7348602099438395, P38=-0.5684400267321882, P39=-1.4306633582868207, P40=0.7492487869525126, P41=0.734388858190024, P42=0.7546446894513128, P43=0.7296982894007672, P44=0.6739050578282031, P45=0.6296694796918204, P46=0.6552072122434975, P47=0.6562709240028157, P48=0.6540991905539394, P49=0.7290465355632312, P50=0.716054472242452, P51=0.7250339641996362, P52=4.668182314878859e-05, P53=4.852925301583004e-05, P54=6.249488076234125e-05, P55=0.00011419596825283011, P56=0.0009588235088236445, P57=0.01108358360790789, P58=0.024102796121902335, P59=0.06670983544679637, P60=0.12556175765571895, P61=0.0886516713913785, P62=0.039841622629518215, P63=0.006561483789810783, P64=0.005024423946409955, P65=0.009098281108594014, P66=0.019695371573349153, P67=0.029485946131321574, P68=0.03675270411671467, P69=0.04774875174163717, P70=0.048027803229758026, P71=0.0199453797343262, P72=0.011575885527916187, P73=0.010503226626322889, P74=0.012628463868897825, P75=0.013122120059423047, P76=0.011140738035288202, P77=0.0396836574625039, P78=0.07163656625495551, P79=0.06042215030832699, P80=0.026109708079008975, P81=0.011700154473285328, P82=0.01136436623619045, P83=0.011978389798014482, P84=0.014074401393236057, P85=0.012496741199947894, P86=0.00837405725051771, P87=0.01669550945527461, P88=0.04078366617277096, P89=0.03657793271356516, P90=0.02765789854518554, P91=0.019012684670648608, P92=0.009662431228179688, P93=0.019357657554731508, P94=0.045047211162767595, P95=0.03281341193686746, P96=0.016742907207540268, P97=0.004657112740220514, P98=0.0020731286196936215, P99=0.0037306333803007277, P100=0.003992227225843405, P101=0.0014526924751333675, P102=0.0015100608551204481, P103=0.0027859073813046647, P104=0.005296584566532571, P105=0.003001689774239745, P106=0.003724178399809346, P107=0.006715968399779002, P108=0.00822426953426114, P109=0.003269963882841828, P110=0.0019196048164051112, P111=0.0022872347247702984, P112=0.001153904176128384, P113=0.0007967830203311021, P114=0.0006574439725856286, P115=0.001142511826862687, P116=0.0002557714678151064, P117=0.0003567115842572285, P118=0.0005385353622960632, P119=0.00024034596222581497, P120=0.0008892401255026463, P121=0.000740777726819147, P122=0.0005325570772730039, P123=0.00012904890520708468, P124=7.644948341171617e-05, P125=7.958723163374274e-05, P126=5.737380345541079e-05, P127=7.117764309181125e-05, P128=6.139803603853294e-05, P129=8.790955233925214e-05, P130=8.598724739105914e-05, P131=0.00010154470810190936, P132=9.589495837604729e-05, P133=0.00016406046674341722, P134=6.744438540308159e-05, P135=2.1726234670412936e-05, P136=2.697623778404537e-05, P137=2.5775297830966784e-05, P138=2.551724658996851e-05, P139=4.678572619389777e-05, P140=0.00010938182232142844, P141=8.069340426032202e-05, P142=9.439038200627623e-05, P143=0.00011534447200961022, P144=9.973285316597255e-05, P145=0.00012573074243869757, P146=8.545439081809678e-05, P147=5.9172462807602485e-05, P148=4.762248180586337e-05, P149=3.9193493410848436e-05, P150=4.922584584442031e-05, P151=8.535591847394051e-05, P152=0.00010659805727859633, P153=8.774448896105087e-05, P154=9.302773699892508e-05, P155=5.587102672384796e-05, P156=5.69500974984554e-05, P157=5.9367227394061625e-05, P158=4.9509923973425416e-05, P159=5.5003154772959994e-05, P160=4.710780216965073e-05, P161=3.4133986998489456e-05, P162=3.144715880248271e-05, P163=3.1597919786418995e-05, P164=1.9501985791328137e-05, P165=1.8076929712877595e-05, P166=3.242277008191598e-05, P167=5.0383642438971896e-05, P168=5.3286626920461336e-05, P169=4.593016866520329e-05, P170=1.9812452652843962e-05, P171=2.11636510475646e-05, P172=1.7428733600671258e-05, P173=3.09923615325624e-05, P174=3.103708217379698e-05, P175=3.942044840039698e-05, P176=4.055535011080861e-05, P177=3.1358493082680826e-05, P178=1.4703989432350167e-05, P179=1.2858900417558393e-06),Row( P0=-514.3544106513845, P1=32.69835868420853, P2=-11.064660920811235, P3=-2.6306896070028394, P4=-7.292507335022539, P5=-7.5374954128758125, P6=-11.697534279885831, P7=-3.134077301900703, P8=-1.3274019782946642, P9=-2.18886704798832, P10=-1.3281401203364123, P11=7.541820803353403, P12=7.357205834611129, P13=8.429627992232678, P14=10.62292084632996, P15=4.798553734801387, P16=3.29531320236215, P17=1.8458778484211247, P18=0.8027553856907425, P19=4.336325785569603, P20=1.0202434595620122, P21=5.793550187394802, P22=1.611486796845567, P23=4.220873210443932, P24=2.520290374473413, P25=2.8363342631083044, P26=2.737880384057382, P27=4.122840434007011, P28=1.5160921318077485, P29=0.4327222222461547, P30=-0.1666758668707531, P31=4.374068839842211, P32=1.4444909414001899, P33=0.9936268815153743, P34=0.22528361247778383, P35=2.505436501527668, P36=1.852157188199724, P37=2.1634841264680795, P38=1.1226938412980527, P39=1.1372826128830644, P40=0.617163282376452, P41=0.6253984007037441, P42=0.6910532014789228, P43=0.6554464092709036, P44=0.6612560591831796, P45=0.6739537490928514, P46=0.6544492519086768, P47=0.6043042789689684, P48=0.6135281512690671, P49=0.63006665271556, P50=0.6210243507915438, P51=0.627416903310017, P52=0.021498546656531087, P53=0.0017232828801142276, P54=0.0005042078398505671, P55=0.00017384639332968058, P56=0.0001938985252631347, P57=0.0018147486926660792, P58=0.010392462531774133, P59=0.019942850166660347, P60=0.036644505483242294, P61=0.1016390634432321, P62=0.22530951360590717, P63=0.192504884523801, P64=0.130502135527725, P65=0.1642167709891113, P66=0.12707697650069835, P67=0.08343317139800667, P68=0.11636167708696113, P69=0.38151251559310834, P70=0.9225915741488608, P71=0.9320187838208884, P72=0.6854997401904568, P73=0.7681779967486237, P74=0.7829356730756525, P75=0.8822208379605908, P76=1.1978012968121066, P77=1.5533201658561517, P78=0.47791236720870134, P79=0.1271890816813622, P80=0.10582519663764689, P81=0.1119142411062344, P82=0.11266055595178735, P83=0.13643803924465547, P84=0.16842811468187585, P85=0.12310234225735027, P86=0.09599155991587498, P87=0.1315160626914771, P88=0.7527023738830855, P89=1.0466805477328804, P90=0.596568039232808, P91=0.4123890728509, P92=0.5547849422225032, P93=0.596068672837082, P94=0.1740315879623976, P95=0.021398312323342748, P96=0.020790467214050118, P97=0.03100998473362729, P98=0.061109833652929685, P99=0.12331634145391401, P100=0.1190896220651068, P101=0.116071255841378, P102=0.20980443505636945, P103=0.1090299451561777, P104=0.1349953239220649, P105=0.16510245251437553, P106=0.11062852897853238, P107=0.024787965800598173, P108=0.033221379420152714, P109=0.029262999593383576, P110=0.034326429970562525, P111=0.028099124315683188, P112=0.047226462072406473, P113=0.052013567914809676, P114=0.09684733763506032, P115=0.15517606932807182, P116=0.04458533917215468, P117=0.035229023917438665, P118=0.013811335600111864, P119=0.02266720206951987, P120=0.01102877003719616, P121=0.006119944221824047, P122=0.0059868223678975286, P123=0.005811146711932966, P124=0.011847179238810096, P125=0.023843520952043076, P126=0.058021833524522004, P127=0.0034477534447112006, P128=0.001077016376554552, P129=0.003658480973205731, P130=0.00359061496719941, P131=0.0024965928097398036, P132=0.0023642402443772564, P133=0.002282395110583598, P134=0.0020838784876100443, P135=0.002716575254484171, P136=0.008762533256024661, P137=0.0022986664600474684, P138=0.000950563038237351, P139=0.0014478153938141305, P140=0.0020404830293577992, P141=0.0014443188087385828, P142=0.0029040810807375466, P143=0.0021269110381165013, P144=0.0019810455117533405, P145=0.002947904992171324, P146=0.0022689176909468003, P147=0.0030893551945069056, P148=0.0019924725215920247, P149=0.0022568633803290087, P150=0.0023975043649241985, P151=0.0027287544830243158, P152=0.0019805037762481987, P153=0.0021307352185245304, P154=0.002614252957677289, P155=0.002547358575832894, P156=0.0031406833649522087, P157=0.003758495817894787, P158=0.0034816793380240507, P159=0.00515903544868285, P160=0.004844240674407825, P161=0.0047721308646761365, P162=0.005004625595945572, P163=0.0043587942115518264, P164=0.004612543517415629, P165=0.0021664998134202124, P166=0.0021356745461945975, P167=0.0027429475827363448, P168=0.0027635669180931053, P169=0.002074344421555764, P170=0.0024722919028080975, P171=0.0018025720642654394, P172=0.0011707163376460728, P173=0.0019127643519027543, P174=0.0018083404320319192, P175=0.0012120460024780896, P176=0.0013914027893946973, P177=0.0011932063474479429, P178=0.0006311436004432981, P179=7.214453164298808e-05),Row( P0=-598.7498215239598, P1=51.88443576565119, P2=2.490019122551634, P3=10.750352277470235, P4=-0.3534013816507383, P5=-2.7537114808054532, P6=-1.5281696485939826, P7=0.2540726570304805, P8=4.952348025798439, P9=-0.6006600528986097, P10=-4.097010813734323, P11=3.5975735537954456, P12=-4.189627099337999, P13=-1.4705822733170826, P14=1.967137279338924, P15=-2.37436904657874, P16=0.8271322673367559, P17=0.9299328763963693, P18=-1.828134194379307, P19=-1.478517292057808, P20=-3.0605791863228924, P21=-1.892966849986064, P22=0.2703576922090595, P23=0.9029858232170048, P24=-0.2779891039361845, P25=2.0946733413946768, P26=0.2400798446634343, P27=1.5901841822870517, P28=-0.41958181185517474, P29=0.1576029537354265, P30=0.03670058234378027, P31=4.134698784164273, P32=2.89877561762781, P33=4.129118181248868, P34=0.982273281675992, P35=0.9841083754723164, P36=0.1920042799277011, P37=2.8799033377729177, P38=1.5222887826030111, P39=2.352958766196607, P40=0.7892680381872452, P41=0.7113934166764821, P42=0.6995841484650192, P43=0.6784009317223403, P44=0.6733188082308593, P45=0.6808822144313849, P46=0.6805824222360465, P47=0.688313436482481, P48=0.7521627374837477, P49=0.7470559324321758, P50=0.7431920587245034, P51=0.7908859055173871, P52=0.00029839017628211526, P53=0.00016246561806245176, P54=0.00012531868697099988, P55=0.0001379271525936233, P56=0.0005079119158730388, P57=0.004320565924601391, P58=0.016824071846268773, P59=0.05262063709424395, P60=0.06301573027648816, P61=0.09539300353029179, P62=0.13868873765399664, P63=0.041738625595287605, P64=0.004627304837075854, P65=0.01089988849991194, P66=0.023072083010109195, P67=0.04294803504156921, P68=0.04730733608025548, P69=0.04028182615033506, P70=0.07252265280981003, P71=0.05358007237673554, P72=0.06521310252155343, P73=0.09627811169142805, P74=0.04547075295443789, P75=0.008664926447696264, P76=0.0071410247396908105, P77=0.01514396057943195, P78=0.023553015660845826, P79=0.05138569901228548, P80=0.04016798239154062, P81=0.030895455698805675, P82=0.07403971587373329, P83=0.18701563845372393, P84=0.20445964310796513, P85=0.07116479420523268, P86=0.014141987595524718, P87=0.008155399890471681, P88=0.009963619838931096, P89=0.004113119985699166, P90=0.0034041643773025467, P91=0.05610200146143451, P92=0.464456142651701, P93=0.3608053029278462, P94=0.0278217656898526, P95=0.002194574993808746, P96=0.0016778447269928485, P97=0.003070426871082499, P98=0.014238331369322395, P99=0.05062057297265606, P100=0.035284139021808235, P101=0.0033263086673089412, P102=0.000653048479483344, P103=0.0016390520886462579, P104=0.02961222660120588, P105=0.1162003578073165, P106=0.06648413963131952, P107=0.0030450099010499846, P108=0.018918697828851846, P109=0.035495303544032304, P110=0.0358694962556334, P111=0.03143635257531009, P112=0.01659510364282197, P113=0.009806089335989539, P114=0.006231467667570027, P115=0.014262210949512704, P116=0.010036238754077727, P117=0.0016408633941574163, P118=0.006028523542046074, P119=0.00572650712935242, P120=0.0019286121059956427, P121=0.002658231595483305, P122=0.005843875456389978, P123=0.0013290461743940953, P124=0.0017795614080497425, P125=0.0015203344225313779, P126=0.0014065762265776085, P127=0.0021840950139004225, P128=0.0010305673753777088, P129=0.002395601868752158, P130=0.002622996958927919, P131=0.0005335565162924041, P132=0.0006026737395445304, P133=0.0011954171121228404, P134=0.0004656643509120848, P135=0.0004769654590974501, P136=0.00043322307363966587, P137=0.000517687688973043, P138=0.0003549190596392584, P139=0.0009068812553815342, P140=0.0014095751457455553, P141=0.0023248112855202884, P142=0.004591159800326623, P143=0.0034868425753441634, P144=0.003503531105928664, P145=0.002753073669444893, P146=0.0018973427410802163, P147=0.0010334607977497913, P148=0.0007116963483761214, P149=0.000472306584270686, P150=0.0004559278691606976, P151=0.00041064945692971343, P152=0.0005810606171399321, P153=0.00045805663439828784, P154=0.00017530966267144393, P155=0.00015497974363482152, P156=0.0002652864221023499, P157=0.00037429543495932026, P158=0.000545889249309621, P159=0.00042555175269818886, P160=0.00032805416518128856, P161=0.0003881016143531152, P162=0.0005503218759399256, P163=0.000684560984408819, P164=0.0005577205860779356, P165=0.0004844551601405508, P166=0.00043424734968637964, P167=0.0005669736236593947, P168=0.0004884961758215438, P169=0.0005152145611837354, P170=0.0002448424546320072, P171=0.00020063732358760296, P172=9.352977645159497e-05, P173=0.00010967259513911882, P174=0.0001470634229402842, P175=0.0001272283296000923, P176=0.00010511911538692482, P177=7.094954493924027e-05, P178=4.602787945085314e-05, P179=7.601180224947535e-06),Row( P0=-662.8620752460997, P1=74.08847614673452, P2=11.31974968677818, P3=15.911334678222083, P4=3.930708938954976, P5=-0.8492740563197807, P6=1.5016656416734044, P7=2.154846515285759, P8=7.030079469201794, P9=1.7747425165198711, P10=-3.549313435301841, P11=3.4678804164858557, P12=-1.999995074319307, P13=0.08090011164934698, P14=2.494238503995462, P15=-3.468391184278331, P16=-0.3457132132980301, P17=-0.3272973681931991, P18=-1.4922265345619683, P19=-1.398273387587511, P20=-2.8718900016275515, P21=-2.463936647236316, P22=-2.575522662501623, P23=-0.23323648351789286, P24=-1.7812132799440004, P25=2.7211755296381797, P26=0.4274238327791089, P27=3.7343400891062686, P28=-0.06677716442283786, P29=1.5342239263799327, P30=0.10654656274489745, P31=2.8977152781743225, P32=1.5549243334119258, P33=3.7533887598560067, P34=0.27640850292019464, P35=4.915690202956255, P36=-0.12137995059983869, P37=-0.454717172754887, P38=-1.7408971844321717, P39=0.7265680592223195, P40=0.7565889214059996, P41=0.7205583573467105, P42=0.7258148298908302, P43=0.7497515412761774, P44=0.7223807385578921, P45=0.6531934235890339, P46=0.653314130310737, P47=0.6750973492835137, P48=0.6412240653656199, P49=0.6579138990964486, P50=0.6985274328345782, P51=0.736019894908129, P52=0.00011757379420438262, P53=5.8028079857753684e-05, P54=4.0621825261327756e-05, P55=5.8876101670526785e-05, P56=0.0005947474232863197, P57=0.005707570021914167, P58=0.019112919484186884, P59=0.0663145779887046, P60=0.08592780022018355, P61=0.11185690628378346, P62=0.052798449789308234, P63=0.007587639831554948, P64=0.002772121980651352, P65=0.005359077253273197, P66=0.017489824533321385, P67=0.0396716825045809, P68=0.026670136981255883, P69=0.013531837732078876, P70=0.029439369485428458, P71=0.03146553231027391, P72=0.025558038688949666, P73=0.015349275400360145, P74=0.013124636341999719, P75=0.026093486300034732, P76=0.03223063669080913, P77=0.024851638457661515, P78=0.012738887072552081, P79=0.01140178248498001, P80=0.011583855654361432, P81=0.01760192445834243, P82=0.033608518614469916, P83=0.06964110840694791, P84=0.06330415924717382, P85=0.02223187390066167, P86=0.00695429768981485, P87=0.005728805181480962, P88=0.016018735655393253, P89=0.0421367185481285, P90=0.07122239122453307, P91=0.037849853056629704, P92=0.008047530674237595, P93=0.0017590366678417989, P94=0.004103299464213249, P95=0.02552452826967868, P96=0.022169285293220403, P97=0.02132668007799566, P98=0.011935994883993004, P99=0.0012398357899742498, P100=0.0024307152054572675, P101=0.004140252540910456, P102=0.003978544265660758, P103=0.003581560436488252, P104=0.003064384084409895, P105=0.0014877137072801117, P106=0.0020334002252236287, P107=0.00809347535567971, P108=0.008602778018391107, P109=0.003163283734588192, P110=0.0025165488292031605, P111=0.003996282318309618, P112=0.0009560706613944315, P113=0.00042196612509916043, P114=0.0007641574611680225, P115=0.0007207337771142418, P116=0.0004926083522348139, P117=0.0006099989129803799, P118=0.00025723000227953333, P119=0.00014530946061163498, P120=9.58121909361177e-05, P121=0.00015369369911206375, P122=0.0001249564129369315, P123=8.100875981417793e-05, P124=7.482631586026126e-05, P125=7.866395051117612e-05, P126=0.0001265232357310835, P127=8.408027906798214e-05, P128=9.552401994580093e-05, P129=3.5109791898921104e-05, P130=5.770824596937996e-05, P131=8.158714146299037e-05, P132=7.34759162964073e-05, P133=2.538015428433917e-05, P134=3.095428392345248e-05, P135=1.7343351078595174e-05, P136=2.1995620047591174e-05, P137=3.858711622443585e-05, P138=5.757863148541629e-05, P139=7.095700238613406e-05, P140=7.319349730128144e-05, P141=9.420787620646237e-05, P142=7.858639801898996e-05, P143=0.00010203355375989581, P144=0.00012160869315585161, P145=0.00010852172860867586, P146=0.00010059973896058091, P147=2.678650512351682e-05, P148=1.602822095184273e-05, P149=1.4641355377582575e-05, P150=1.148818050809275e-05, P151=1.772147809448385e-05, P152=3.750435165535041e-05, P153=6.655455940267802e-05, P154=6.633978682907369e-05, P155=5.359901385381597e-05, P156=3.2542422710585774e-05, P157=2.139142881694317e-05, P158=2.9570559712361392e-05, P159=3.30034493002009e-05, P160=2.229019842174623e-05, P161=4.012599004898985e-05, P162=2.753290500871066e-05, P163=2.2345255766721286e-05, P164=1.5234558988447742e-05, P165=1.0047765017707174e-05, P166=1.0941472786401416e-05, P167=1.0757940991190267e-05, P168=1.4402502789485116e-05, P169=1.9799014516999674e-05, P170=2.3530588828227015e-05, P171=9.63382357010018e-06, P172=3.1502813916106808e-06, P173=6.971771442353864e-06, P174=8.834688313262007e-06, P175=7.604723343279816e-06, P176=6.493240528684291e-06, P177=4.679108949999269e-06, P178=2.7788872834721488e-06, P179=2.9075473846097144e-07),Row( P0=-559.5517885347157, P1=41.43289970283025, P2=4.302625728185224, P3=2.212529151076001, P4=-4.21653924313205, P5=-3.4247597375362955, P6=-6.136750629593076, P7=2.2589989797142587, P8=0.7103281384407105, P9=-3.819669308151757, P10=-7.420644825883305, P11=2.3710292613984496, P12=-4.713511289936179, P13=0.6901340272660925, P14=5.941939046300962, P15=4.262355312802728, P16=9.058624061778625, P17=6.019628143940551, P18=5.2862124296647846, P19=3.2730625105181144, P20=1.477398665795669, P21=0.6107941282475133, P22=2.865055698022065, P23=1.8045804567790484, P24=-0.25183401239099645, P25=3.152135702004737, P26=-1.288787276403121, P27=3.417976458183558, P28=1.6304109181248316, P29=2.2165591992327522, P30=-0.35533348339960635, P31=1.7505829654708076, P32=1.3956545574017647, P33=2.2904222774551286, P34=0.541101400231417, P35=2.4611101908061914, P36=0.7917500215757822, P37=1.3307778441193443, P38=0.27430757115292215, P39=0.6123567272126402, P40=0.7425263672982744, P41=0.6928285995781479, P42=0.6577316392677222, P43=0.654846612834514, P44=0.6231181341970562, P45=0.6156274509062699, P46=0.6233447151968764, P47=0.6256018033314117, P48=0.6540733257006501, P49=0.6950950167402001, P50=0.7211103423477022, P51=0.7275558808405375, P52=0.0008008314883460096, P53=0.0011674473504888045, P54=0.0003836790404125542, P55=0.00025096136229773534, P56=0.00026024390340038234, P57=0.0010868103038850533, P58=0.003734981256938216, P59=0.012620396586194084, P60=0.031794331465300606, P61=0.038782541585511046, P62=0.07029818440097366, P63=0.0868655911884039, P64=0.0827808820246695, P65=0.10213485301295819, P66=0.11431399611373018, P67=0.22857895620484378, P68=0.2507142565460693, P69=0.6174883628860007, P70=1.348399761364492, P71=0.45126435940742415, P72=0.004504163172091515, P73=0.004853087644374899, P74=0.006445860949569989, P75=0.005905187214910599, P76=0.005396334821383581, P77=0.011655881432840486, P78=0.029368138369369603, P79=0.0425965853488696, P80=0.028375114652891404, P81=0.03501458509685453, P82=0.07814535748313545, P83=0.22933310435257329, P84=0.28014375531863867, P85=0.1343744360607111, P86=0.05409684516035924, P87=0.07802389175561364, P88=0.3210729221830232, P89=0.15987538229767206, P90=0.013234150636397485, P91=0.009602302282284524, P92=0.013303488315226392, P93=0.009343427371040016, P94=0.010923718178364983, P95=0.013119004596072925, P96=0.017531928487160616, P97=0.01884753977710701, P98=0.01414445598420409, P99=0.01392492141851743, P100=0.023326108112426757, P101=0.02875689780418547, P102=0.00221835374487656, P103=0.001321161071596659, P104=0.0018905201124749427, P105=0.0033069323630730693, P106=0.004728769687319728, P107=0.0015662425600928831, P108=0.003382718939496793, P109=0.004518166133055476, P110=0.0011067518088982936, P111=0.001616284386389423, P112=0.0011262675883105919, P113=0.0012372472702856779, P114=0.0012059604909319165, P115=0.0015600744266696725, P116=0.002280831260467538, P117=0.0030812981248508954, P118=0.0009153845560039496, P119=0.0006770348849551806, P120=0.0007199887333548294, P121=0.0010119414967687894, P122=0.0009827628463258168, P123=0.00142868663512883, P124=0.0010502837671580163, P125=0.0011424707801336647, P126=0.0010174414515348985, P127=0.0004293518108979588, P128=0.00011643788106146137, P129=0.00010537512490825407, P130=0.0001467812703421872, P131=0.00015043598087938523, P132=0.00013690356351879268, P133=0.0003668068115148411, P134=0.00018608337232475696, P135=5.815061387999249e-05, P136=5.1314966161450164e-05, P137=6.617534574856847e-05, P138=6.857572966596971e-05, P139=7.454431320092643e-05, P140=0.0001528847914791452, P141=0.0002480338988394192, P142=0.0003896839687834325, P143=0.000579024853581521, P144=0.001016184017711565, P145=0.0006954553064270887, P146=0.0008317545815325554, P147=0.0008368702405328713, P148=0.0007665318126861018, P149=0.0007564307850898074, P150=0.0005703845558641069, P151=0.0005919436807220848, P152=0.00046281955814866677, P153=0.00044308619617159934, P154=0.0004053919554195991, P155=0.0004663201901669573, P156=0.0004335868317898098, P157=0.0004828880283110371, P158=0.0005260704636576788, P159=0.0011998155801324023, P160=0.001440411534209258, P161=0.002175220763042662, P162=0.0026053603049370107, P163=0.003443374835152904, P164=0.0031882313935512295, P165=0.003944632131942129, P166=0.004739423304480551, P167=0.002586181689089934, P168=0.0013957998425518272, P169=0.0010076335785188786, P170=0.0005300592820324233, P171=0.00027891741396384313, P172=0.00020147560247999136, P173=0.0001924575393191589, P174=0.00016366298666047304, P175=0.00016798587289423123, P176=0.00016309858873926288, P177=0.00017551508212473657, P178=8.504044043598608e-05, P179=1.098311716976926e-05),Row( P0=-577.8351758592025, P1=58.100821604393474, P2=-4.841660640346913, P3=13.581597315098106, P4=5.717100774349098, P5=-5.207382879419027, P6=-4.715510850214185, P7=5.348812715923117, P8=11.87713533650933, P9=1.243057478510447, P10=-4.119446602993876, P11=6.588694898876973, P12=2.812698281141007, P13=-1.2789097078492355, P14=4.857062864293166, P15=1.7411884663861361, P16=3.3627777722975294, P17=3.7254662258132916, P18=0.8096736854140043, P19=5.601754638371561, P20=5.000273083596777, P21=5.403118759445233, P22=2.438259756865375, P23=4.22120938846243, P24=1.7877983531376345, P25=3.6676233074818354, P26=1.5413764301795851, P27=6.717683587734522, P28=6.024941361841802, P29=6.0311247720135155, P30=4.583618618868367, P31=7.6646480530342975, P32=4.56864808096435, P33=4.581070940734825, P34=-0.6488856814113918, P35=1.6586286816913343, P36=2.441900697966186, P37=3.864988538604551, P38=2.384407989023775, P39=1.411161849659237, P40=0.6442586875186919, P41=0.6538172551123022, P42=0.6377449779324722, P43=0.585456165177836, P44=0.5858733160402794, P45=0.6780460878054927, P46=0.7507679944521273, P47=0.7153167903641333, P48=0.7128735170440876, P49=0.7417737401169946, P50=0.6630367517806594, P51=0.6201504291997824, P52=0.06339245628931918, P53=0.006330586528135557, P54=0.0005022975378384754, P55=0.00039408103255951304, P56=0.0003541078180081481, P57=0.0007749125889329673, P58=0.005801891658582885, P59=0.058468858057273095, P60=0.17327135231116605, P61=0.16210351915210675, P62=0.32255249052846047, P63=0.4277005597641072, P64=0.27002024820932646, P65=0.3011277288458999, P66=0.15095306239741302, P67=0.03240006372553115, P68=0.049246087939320726, P69=0.09678851558691277, P70=0.10080815787171946, P71=0.033480656311236555, P72=0.010880294991924477, P73=0.013516432861641154, P74=0.040332790143232614, P75=0.0440454859477289, P76=0.03773441767750392, P77=0.11346555122719441, P78=0.28084944821512503, P79=0.3578461769687185, P80=0.15821261353434765, P81=0.03501159924070348, P82=0.01019450908322276, P83=0.01834837204035176, P84=0.1138210193378509, P85=0.15104378067282395, P86=0.07924954161966946, P87=0.1027625700149952, P88=0.22164963142536345, P89=0.10557456068552332, P90=0.10434556043607979, P91=0.13309892896673667, P92=0.34465935226525196, P93=0.24427713297088804, P94=0.11426197231123134, P95=0.1280238629407036, P96=0.01374425634190966, P97=0.010152672701294142, P98=0.02372825999242214, P99=0.035989230993484764, P100=0.052062560158184265, P101=0.12225770627370783, P102=0.0698147511584603, P103=0.007216626806517463, P104=0.003619619062587621, P105=0.01416545570551578, P106=0.02003215431738552, P107=0.04718741836988429, P108=0.19190531197722283, P109=0.10664603407330137, P110=0.008298693868332025, P111=0.009028090589595585, P112=0.028096273573465377, P113=0.06384245964260794, P114=0.00938104837597744, P115=0.003960832667884725, P116=0.008460169318711139, P117=0.0032208939103662445, P118=0.004111800015456117, P119=0.0056068220947661125, P120=0.0029400150163766147, P121=0.0021962044442696096, P122=0.0020618115395519164, P123=0.0014449753747037988, P124=0.0013058538510440983, P125=0.002061347094917614, P126=0.0021093956487615545, P127=0.0006444384799241419, P128=0.0011025440185794795, P129=0.0011749736151819253, P130=0.0005892394874961735, P131=0.0006756826796500735, P132=0.0010373214420238155, P133=0.00044635036179980207, P134=0.0003483488945264689, P135=0.0004620141294974086, P136=0.0003876414461201319, P137=0.00044961677384384167, P138=0.0004985536255749098, P139=0.0007449878446696558, P140=0.0005419503425271806, P141=0.0006292919187558791, P142=0.0006181090973916423, P143=0.0007904642714519875, P144=0.000947690984342808, P145=0.0005214159284412277, P146=0.00037921471590044093, P147=0.00020121748111503833, P148=0.0002671485986108311, P149=0.0003375606686363971, P150=0.00035459390298927764, P151=0.0002948016811581372, P152=0.00017040763868980089, P153=0.00015819477921940377, P154=0.00018126471190911165, P155=0.00017057880899268733, P156=0.0001938429117959607, P157=0.0002473757166819133, P158=0.0003144114601562674, P159=0.00032171664034182096, P160=0.00030984573327694773, P161=0.00041267252108620836, P162=0.0003500354839747094, P163=0.0002514207602315909, P164=0.00011406872284426222, P165=0.0002068507916689888, P166=0.0001407573374311666, P167=0.0001444145447236548, P168=0.00012572868215303013, P169=9.03104050097534e-05, P170=4.9189835267754274e-05, P171=3.8548894653702064e-05, P172=4.493248586233088e-05, P173=6.022177174981833e-05, P174=4.967266048962379e-05, P175=5.298500260528631e-05, P176=5.6136490076327975e-05, P177=9.232743394827121e-05, P178=5.280210156819993e-05, P179=6.928813340047277e-06),Row( P0=-693.5870890937546, P1=87.23026616215932, P2=13.389270727635024, P3=18.210553739734433, P4=6.408371385804994, P5=5.048189477525399, P6=-0.49634252053468436, P7=0.6063668649996985, P8=9.417534566696057, P9=0.11606903241054795, P10=-5.783100302492331, P11=3.7192549277165785, P12=-2.5044641163041, P13=1.7914479029130181, P14=2.4500088636838275, P15=-4.179045171627948, P16=0.33902160313036817, P17=-1.9040277709202558, P18=-1.7243748991063699, P19=-1.3167527188100199, P20=-0.6629212885417772, P21=-2.3434392135073416, P22=-3.2449193567191625, P23=-1.7910717611188285, P24=-6.113299160855733, P25=-1.4955886563744154, P26=-4.7707710555969065, P27=-1.4662851850818739, P28=-4.713856386640761, P29=-2.2986584074856773, P30=-4.377423589768577, P31=1.467727174039117, P32=1.5367685378646287, P33=5.709203369560179, P34=5.151130402748743, P35=9.529792015886095, P36=7.525744437867576, P37=6.140398250162975, P38=2.030773683035444, P39=1.6797994393244033, P40=0.666073110225798, P41=0.6553453681537689, P42=0.6334068397656709, P43=0.6300697135540966, P44=0.637604496102634, P45=0.6462164953794494, P46=0.6556507264527732, P47=0.616632540174549, P48=0.6650521939666342, P49=0.7876700158268497, P50=0.8286757667061202, P51=0.7685024893213, P52=7.091535478952216e-05, P53=4.774995815837799e-05, P54=3.3302003976645464e-05, P55=4.5045695177962314e-05, P56=0.00034657842016319373, P57=0.008648514332793733, P58=0.04887317351942185, P59=0.07127734553771306, P60=0.017163478942885, P61=0.0014315585926855344, P62=0.0008007814678251999, P63=0.0010339381084467879, P64=0.004297064628449646, P65=0.048328603902938055, P66=0.07107955735544241, P67=0.042370532717195805, P68=0.011697151811099614, P69=0.0008781377596222674, P70=0.0008938404515517964, P71=0.0019858202720454436, P72=0.005859978436174472, P73=0.011113653485204452, P74=0.017423267810058513, P75=0.01668227660558503, P76=0.004658330228732345, P77=0.001014330359723882, P78=0.0014505074520925238, P79=0.003626208001922994, P80=0.0094211265089894, P81=0.018627750066356644, P82=0.01802176581775331, P83=0.011421636481306927, P84=0.00397618606447304, P85=0.0018428076605714698, P86=0.002264746405411645, P87=0.003424214371995472, P88=0.008879830154492407, P89=0.019662290082575362, P90=0.026421024410420227, P91=0.008254049062139501, P92=0.020932545666330543, P93=0.007687098177983134, P94=0.002335396930878901, P95=0.012516148878407728, P96=0.0055910777550005, P97=0.004989028726182331, P98=0.0008732938612586964, P99=0.0006598072256387268, P100=0.00269729572684607, P101=0.0020285829864709636, P102=0.0003637595595046696, P103=0.0006748728087729486, P104=0.0024309568591607826, P105=0.0007284420786498046, P106=0.0002146193328844226, P107=0.003193205945028703, P108=0.0065060233220905145, P109=0.0011684362891812036, P110=0.00046582426679055833, P111=0.0021830312765132947, P112=0.0012591000346329946, P113=0.0004081817924557303, P114=0.00045414096258396074, P115=0.0001584411584333979, P116=0.0006622646581232468, P117=0.0002445298154608076, P118=8.996975110025201e-05, P119=5.641638680045597e-05, P120=7.987713193902703e-05, P121=8.567143661766051e-05, P122=7.788495024498939e-05, P123=2.7576420028216948e-05, P124=3.0373556337840558e-05, P125=3.2021446081500464e-05, P126=4.308069291368495e-05, P127=2.4212456841025883e-05, P128=1.9335659817181012e-05, P129=1.8553072561743174e-05, P130=2.5880374984545313e-05, P131=3.569013383006226e-05, P132=1.5949189511797262e-05, P133=1.4482201982034886e-05, P134=1.102322961364361e-05, P135=4.709218600925374e-06, P136=7.022398175522916e-06, P137=6.9653286652160884e-06, P138=9.84023706768233e-06, P139=3.1896751580019026e-05, P140=5.9271986910475064e-05, P141=5.1319591065928116e-05, P142=0.0001158199488848357, P143=9.892769685380397e-05, P144=7.218195214393412e-05, P145=5.4179046841151615e-05, P146=3.536200842906339e-05, P147=2.5196057133524563e-05, P148=3.424990849534745e-05, P149=5.140725075218408e-05, P150=5.380344074644568e-05, P151=6.681561347106268e-05, P152=9.00662133734107e-05, P153=5.918156294533132e-05, P154=4.573977649355589e-05, P155=4.888968584591062e-05, P156=4.702296126377911e-05, P157=5.3339602511289625e-05, P158=5.556312380805956e-05, P159=5.479820225167916e-05, P160=5.585202543914265e-05, P161=4.904441283052209e-05, P162=4.9080884462537705e-05, P163=5.160897822912471e-05, P164=3.674347425929194e-05, P165=3.6213837305803014e-05, P166=4.259260002759663e-05, P167=4.9537432522744146e-05, P168=4.8809311633606513e-05, P169=2.746721449535934e-05, P170=1.4981329545016991e-05, P171=8.289917317100055e-06, P172=4.6898379091972635e-06, P173=3.8125451801215788e-06, P174=4.31141798276212e-06, P175=6.57482901152049e-06, P176=1.2305866149286601e-05, P177=8.801878772592835e-06, P178=5.242501426122282e-06, P179=6.010502611556923e-07),Row( P0=-651.8707989475629, P1=76.73579742202145, P2=10.27766276480064, P3=22.46142780787165, P4=4.938959923686535, P5=-2.274834440573848, P6=-0.1323257061672799, P7=0.2567774093242745, P8=8.603229320510534, P9=-3.7439135149665934, P10=-3.1127198604648423, P11=5.1521675627405905, P12=-1.75846853216781, P13=0.2207761219244843, P14=2.715927647739352, P15=-1.8189254014597904, P16=-0.2224192341747925, P17=-1.9746732304055241, P18=0.6585676663254805, P19=-2.2990307116102735, P20=-2.885389986627536, P21=0.3219312819565759, P22=-4.409669841047642, P23=-2.135970377275462, P24=-2.8073530888334295, P25=0.002505994784670377, P26=-2.7908910983759485, P27=4.102057343743356, P28=-1.636670977401605, P29=2.695040263360949, P30=0.5890209033356536, P31=4.812163771174819, P32=2.406743312939017, P33=5.2446170813371005, P34=-0.5735242798336343, P35=2.7325634253982707, P36=1.8025278832747134, P37=3.6208147853131467, P38=1.525776536496479, P39=1.907951491461949, P40=0.7557031360705254, P41=0.6993718335552788, P42=0.6559893397002265, P43=0.6300448919029235, P44=0.6449136479506837, P45=0.6322796561414009, P46=0.6485826685479035, P47=0.615945928946104, P48=0.6650061971538994, P49=0.7124667326300747, P50=0.731232845667929, P51=0.7313468373970495, P52=0.00045784389780206494, P53=0.00012104051758039587, P54=4.649874083003262e-05, P55=5.884737335838034e-05, P56=0.00018371165429649101, P57=0.004877551962918564, P58=0.03209840015038891, P59=0.08717598826494773, P60=0.08780997388537425, P61=0.07348444404751227, P62=0.035836300199253324, P63=0.004548426425803887, P64=0.0021277253871794665, P65=0.015053666557586646, P66=0.02954182092802067, P67=0.03152455954927541, P68=0.027636080234105437, P69=0.020577419650693683, P70=0.0332073117702966, P71=0.02829714238943708, P72=0.02501736674946172, P73=0.016382976374882608, P74=0.013806174523261572, P75=0.013446455117388465, P76=0.011153259457971071, P77=0.012004921563398292, P78=0.011400723418837865, P79=0.015220237079605538, P80=0.015103617872084642, P81=0.014775229592917622, P82=0.014992538936950987, P83=0.009064922084741528, P84=0.015186987405553309, P85=0.0168701906882702, P86=0.005168968993112178, P87=0.0018763547498408034, P88=0.002846393397414382, P89=0.006618147425637626, P90=0.007925423017939286, P91=0.022086213761942978, P92=0.03458672972430167, P93=0.004874975561168482, P94=0.0054083709336924125, P95=0.009132618718454562, P96=0.005520342629142072, P97=0.005979075782938644, P98=0.0038060840946052615, P99=0.003563003519149183, P100=0.0032871158753745825, P101=0.0010150334415083252, P102=0.0020718001164975004, P103=0.0014895116759655919, P104=0.0010494839621420413, P105=0.0007330344980103713, P106=0.0017169936753261502, P107=0.001690565032369794, P108=0.00039959700180562115, P109=0.0014660172024065444, P110=0.0028540900381259135, P111=0.0005570573111049206, P112=0.00020383782385788947, P113=0.0003480015124687032, P114=0.00024018333547529648, P115=0.0002648634100940507, P116=0.0002560439788899198, P117=0.00022182609438011605, P118=0.0001329161180239717, P119=0.00010814083349188637, P120=0.00011706492554465249, P121=0.00013365998846173095, P122=8.680691082244962e-05, P123=6.63462629027898e-05, P124=6.1761680241494e-05, P125=6.072758906223819e-05, P126=6.365026513564363e-05, P127=8.24605886201353e-05, P128=8.063528118194744e-05, P129=3.8173722014065006e-05, P130=3.230476145777087e-05, P131=2.4907074157504414e-05, P132=2.5875044361811495e-05, P133=4.0515420250319625e-05, P134=4.1867512957232555e-05, P135=2.411808697198966e-05, P136=6.105780329940225e-05, P137=4.711432524211356e-05, P138=4.775335406783156e-05, P139=3.8168800784553886e-05, P140=3.9023136581210746e-05, P141=3.752062567395958e-05, P142=4.905397371192511e-05, P143=8.990156485295802e-05, P144=8.925027535509913e-05, P145=0.0001007467342683358, P146=4.276313227480176e-05, P147=2.1905171571704626e-05, P148=2.0998191992986404e-05, P149=2.3207865731853786e-05, P150=2.1528473601023837e-05, P151=4.182874851465234e-05, P152=6.999211940238315e-05, P153=5.3138245120775225e-05, P154=3.456607111061988e-05, P155=3.764910756434543e-05, P156=4.1441847912071575e-05, P157=5.596299364138117e-05, P158=2.7074128978312943e-05, P159=2.2655389598098583e-05, P160=3.8112313814271725e-05, P161=4.0871489898202e-05, P162=2.9414502273187956e-05, P163=2.4944557388042877e-05, P164=1.065487833573838e-05, P165=5.419929371836048e-06, P166=6.214835919055434e-06, P167=7.384882270783085e-06, P168=8.02595184959315e-06, P169=9.193055364222223e-06, P170=5.353558806974422e-06, P171=2.0901959023599167e-06, P172=2.3676262850106997e-06, P173=4.869136977784443e-06, P174=8.246309723229838e-06, P175=9.461200944601182e-06, P176=1.0061509932021556e-05, P177=7.673843378753647e-06, P178=3.578848797774634e-06, P179=4.746436354453929e-07),Row( P0=-619.9495266085028, P1=63.566198664766716, P2=3.207233629551325, P3=14.585560236814894, P4=3.679411064313791, P5=3.189508170420049, P6=-2.547314515889274, P7=0.53781884124572, P8=10.151627674109383, P9=0.43807676392295364, P10=-4.741712870658018, P11=6.6931570154661655, P12=-4.1539055588540466, P13=-0.32602285435461303, P14=2.7516233010820166, P15=-2.505838183438793, P16=1.7282908249375017, P17=0.3022184085722341, P18=-1.8846767830534348, P19=-1.5440942382074496, P20=-0.9914678678331121, P21=1.3170077491803247, P22=1.2941452153468722, P23=-1.6721787857625445, P24=-2.6678377744974298, P25=1.3303256481981383, P26=-1.6406517042499638, P27=2.088386496668511, P28=-0.6193094061687188, P29=1.6125262220972958, P30=-0.3303093223452317, P31=3.792697307730052, P32=1.3269178158211976, P33=4.475623553740597, P34=-0.45508246513477674, P35=2.0679266671185514, P36=0.65985741756298, P37=2.3625038320350304, P38=-0.49971079495286025, P39=-0.029125460588776637, P40=0.7281895125991275, P41=0.722792080666613, P42=0.7223385766981064, P43=0.6993454771325741, P44=0.7033867981360367, P45=0.704630481584769, P46=0.6894223045509597, P47=0.6630625442616801, P48=0.7506471830632764, P49=0.7870543960094595, P50=0.7807159772414948, P51=0.8050666366151937, P52=0.00016858426567913245, P53=0.0002352241530837363, P54=0.0002632098005959825, P55=0.00024724004037369774, P56=0.00044113916538610076, P57=0.0034457866096903804, P58=0.022833246626240093, P59=0.07243411517441217, P60=0.07586662891613505, P61=0.06411382286661235, P62=0.09971090415249463, P63=0.049553460138627436, P64=0.006189842508274168, P65=0.011253148295589216, P66=0.03558375314623004, P67=0.07478869778181035, P68=0.034400920188724836, P69=0.021859762052059348, P70=0.024756312918613552, P71=0.011518909398477137, P72=0.006902410730456848, P73=0.01101859750552783, P74=0.021523044693782495, P75=0.029637040736330042, P76=0.010414882792661295, P77=0.007342295514023417, P78=0.017188763669259293, P79=0.018876002316002796, P80=0.008019992247285883, P81=0.010821098764724646, P82=0.042053376273875606, P83=0.06786337194016955, P84=0.02970902797755138, P85=0.014881668187785248, P86=0.010341918619087152, P87=0.012612273475470786, P88=0.012790656825534577, P89=0.0059019184544174125, P90=0.004301532692579967, P91=0.004534819308237681, P92=0.005596890231235885, P93=0.005865228590189793, P94=0.01533095016470097, P95=0.009752845275553724, P96=0.0024187099400319466, P97=0.0027579637504604147, P98=0.003100677538908525, P99=0.007894783321041623, P100=0.013922496721816528, P101=0.006390620508805686, P102=0.001905810926624265, P103=0.0015202491196148096, P104=0.003270926892954964, P105=0.0017267291847650318, P106=0.001314683994897881, P107=0.002020935197735333, P108=0.003207817304048125, P109=0.0023749605609841534, P110=0.0017405749899030906, P111=0.004839473424634544, P112=0.0023782462601328338, P113=0.0006784362245409271, P114=0.0005797172643061272, P115=0.0009425741778914362, P116=0.00031271805771959215, P117=0.00037363340224559415, P118=0.0004707869658214448, P119=0.00030866331451313617, P120=0.0003845102701499268, P121=0.0005480506375956945, P122=0.0004187110996638086, P123=0.0002726159415045403, P124=0.00013019862005289078, P125=0.00011546401462035913, P126=9.000935100245038e-05, P127=0.00011918234994990387, P128=8.664916473540245e-05, P129=8.36115613984531e-05, P130=0.00012096599177791154, P131=0.00010419334456007654, P132=9.24176908933015e-05, P133=0.00012151906685627852, P134=5.066080675590316e-05, P135=5.400997208131733e-05, P136=4.8690221573001825e-05, P137=5.773405762409167e-05, P138=9.207350512102419e-05, P139=0.00012500816858633892, P140=0.00019762716748374936, P141=0.0007315451334920814, P142=0.0007527801192584143, P143=0.0005028942590158407, P144=0.0004967060711248539, P145=0.0008406083988071689, P146=0.0007838790219547965, P147=0.0005548571283860158, P148=0.0008190533785021999, P149=0.0008234328892473534, P150=0.0006873512525807001, P151=0.0005173307579037543, P152=0.0004093952306279555, P153=0.000323485720364465, P154=0.0002131335170466464, P155=0.00016062162446465938, P156=0.00015191254755061355, P157=0.0002317887193297809, P158=0.0003379750495542233, P159=0.00045885052965805343, P160=0.0005407360056594508, P161=0.0006490312221525723, P162=0.0008163830535621692, P163=0.0008162540095640237, P164=0.0006226582009837652, P165=0.0006738046675744478, P166=0.0008901576892397359, P167=0.0005852489428965585, P168=0.0006209311215493385, P169=0.00042734986474038525, P170=0.000259986896856511, P171=0.0002078964668753536, P172=0.0001971186056022211, P173=0.00017674352408330644, P174=0.00019882849484264054, P175=0.00016940935136153118, P176=0.00011831018949710731, P177=9.919166622609153e-05, P178=5.519437508119099e-05, P179=8.598638187848343e-06),Row( P0=-665.0203809321032, P1=68.71606320882212, P2=2.326877271791435, P3=14.418768756553774, P4=3.7195760856268776, P5=-1.3736376602455074, P6=-0.41843757079220273, P7=1.8989863444322932, P8=6.447351637861862, P9=-0.49677018478982327, P10=-3.6414505080400508, P11=1.5943261551242884, P12=-2.060923235175325, P13=-0.9729196279342962, P14=1.4189102671207903, P15=-4.023076077208376, P16=-1.9287559178982179, P17=0.3782771545437742, P18=-0.931948667093446, P19=-0.5881281208279107, P20=-1.3386253741568073, P21=-2.943107155040067, P22=-1.6987150790393382, P23=0.24672325828366948, P24=-1.8295131528561541, P25=3.6228190580602724, P26=1.006597327805376, P27=4.380057932124472, P28=-0.2101748289967847, P29=1.4134229535351621, P30=0.06834898669543975, P31=1.9689960423682322, P32=-0.43600876154636387, P33=0.44527677017707035, P34=-1.059960607124985, P35=3.653587076497102, P36=3.277868292662567, P37=5.482685577422941, P38=4.718742420505611, P39=4.624363175579567, P40=0.6417061938790922, P41=0.6714423431409327, P42=0.6758034188540252, P43=0.7360649593584223, P44=0.7737833107501926, P45=0.7210983781165539, P46=0.6873751014685522, P47=0.6868876922821113, P48=0.7071980056142668, P49=0.7915515611418625, P50=0.7384966428895712, P51=0.6582143776894874, P52=0.00011428784086687149, P53=4.553110854706538e-05, P54=1.8845792752087652e-05, P55=1.1225939832822078e-05, P56=0.0004918981530764729, P57=0.00886025140966438, P58=0.020097548213402826, P59=0.019547433056817193, P60=0.03181487100006844, P61=0.051082623524811924, P62=0.04266105867500145, P63=0.009919354228005679, P64=0.007589151010578243, P65=0.023961249893903226, P66=0.02550944275780101, P67=0.015990933178870105, P68=0.008761910523933743, P69=0.011591430759005199, P70=0.02878358908891339, P71=0.026025919041535495, P72=0.01839753220331549, P73=0.01878778410288703, P74=0.008886085490942984, P75=0.002155173694491945, P76=0.0012134608796723581, P77=0.001911168815902432, P78=0.011189718848576575, P79=0.043837021375036825, P80=0.03307608432546769, P81=0.011822259251469866, P82=0.006071800744947723, P83=0.008080431914777466, P84=0.02431258914786541, P85=0.023336035132424944, P86=0.017070749158337364, P87=0.00435750084151582, P88=0.002793945852876744, P89=0.002515664349876129, P90=0.009640353670062767, P91=0.03316128802513425, P92=0.033365023321758454, P93=0.07399314653632637, P94=0.022699959878561268, P95=0.006991216078202579, P96=0.005651752998485875, P97=0.0005354855155103808, P98=0.0016167349063145526, P99=0.011892798950650136, P100=0.03439488028094168, P101=0.00552004903488267, P102=0.00023353966762993032, P103=0.0006000550331986956, P104=0.001051203738422624, P105=0.005033920148203214, P106=0.006026485782499928, P107=0.002443629497175469, P108=0.0008415594900685767, P109=0.002260665340343478, P110=0.007332497038238467, P111=0.0030277709705609297, P112=0.0011125772789003438, P113=0.0007521856170101467, P114=0.00028966745483894186, P115=0.0006620954828075402, P116=0.00024732298282046115, P117=0.00043009953380333636, P118=0.0004534154905005591, P119=0.00012651557622727787, P120=0.00019120693619972945, P121=0.0002330370056640499, P122=0.0001252354783903074, P123=0.00030302688126883256, P124=0.00019229529102908098, P125=5.319579130573059e-05, P126=7.303949033917464e-05, P127=6.68295389983099e-05, P128=8.340855221492866e-05, P129=3.168192502419934e-05, P130=5.318361667365706e-05, P131=8.618584128301057e-05, P132=9.95312737099287e-05, P133=2.1177927294510747e-05, P134=2.5500591775233586e-05, P135=1.9532097029581437e-05, P136=1.2704335268786445e-05, P137=2.030712814987254e-05, P138=2.974013587916974e-05, P139=3.0089202949540104e-05, P140=3.398529163218807e-05, P141=5.837622825090324e-05, P142=6.702687510503345e-05, P143=9.249468787764265e-05, P144=9.470510974808785e-05, P145=5.6143059050562286e-05, P146=5.1158900820239204e-05, P147=2.0132489933489645e-05, P148=1.0857182312949215e-05, P149=9.13022930282672e-06, P150=1.1771499266337126e-05, P151=2.457341437079302e-05, P152=8.530870672461674e-05, P153=8.758297879249594e-05, P154=1.8130262605690632e-05, P155=2.5157152142191542e-05, P156=3.476033308815482e-05, P157=1.4909080938787418e-05, P158=1.7245051842576264e-05, P159=1.3000039892317032e-05, P160=1.166345214442134e-05, P161=1.4576864095106527e-05, P162=1.024348387359607e-05, P163=1.4138595275065507e-05, P164=7.742106569220926e-06, P165=6.683837162202454e-06, P166=5.563761700332792e-06, P167=4.106834725015728e-06, P168=4.54576038148234e-06, P169=5.921953566288555e-06, P170=3.920372677518634e-06, P171=2.1361006147968794e-06, P172=1.0466553834487145e-06, P173=3.8196645555686915e-06, P174=8.531651677663147e-06, P175=7.205607359242838e-06, P176=4.681586442925307e-06, P177=4.8335741162601385e-06, P178=2.032451603211909e-06, P179=2.2870697244573287e-07),Row( P0=-548.9090268972534, P1=58.58465769465478, P2=-7.296654267309332, P3=10.647283346772824, P4=1.2568314911697633, P5=-3.2819034324754797, P6=-0.6513324948749958, P7=4.692308282757235, P8=10.376803048324994, P9=2.6074962002291695, P10=-4.698747273888191, P11=5.700628001309633, P12=1.672054461162113, P13=-1.349554433814129, P14=5.66770900771836, P15=0.32279473485569593, P16=2.8935453112522285, P17=2.9049862952714256, P18=-0.32804030963774167, P19=5.471946807669638, P20=4.617765362787432, P21=7.535553401824867, P22=6.17607206696789, P23=7.8637307197819135, P24=3.744539223330598, P25=6.462389908613935, P26=5.297784569343935, P27=9.102921975596702, P28=6.413391857301583, P29=5.171677805194238, P30=1.469201358945922, P31=2.5455564938132618, P32=1.3804845292036807, P33=4.086916199306065, P34=0.060129317387320226, P35=2.010780227422708, P36=-0.107419128669279, P37=0.9089925756204853, P38=0.17734695087758393, P39=1.1508969136738476, P40=0.626194390458206, P41=0.6460864749801555, P42=0.7065154646585886, P43=0.7193549673369032, P44=0.7298497116657156, P45=0.7330133938564602, P46=0.7480969265594519, P47=0.7433547164649592, P48=0.7561408632144653, P49=0.7312222576592472, P50=0.6781877637033201, P51=0.6479528315392041, P52=0.12723757523653065, P53=0.005731361519498626, P54=0.00042043148002084553, P55=0.0003622621157669168, P56=0.00033025797913899973, P57=0.0006329458572695938, P58=0.004428836186158682, P59=0.026855612892784826, P60=0.12975406335500583, P61=0.3757397242790156, P62=0.6532126996550471, P63=0.6270836017969729, P64=0.2929891996712916, P65=0.16427336934138118, P66=0.06443489651679435, P67=0.010471833469852055, P68=0.006606685922659774, P69=0.020399042717913048, P70=0.05554749607963627, P71=0.08768056250208063, P72=0.08414778866781594, P73=0.10210372017469303, P74=0.13534486464450776, P75=0.1508767152959133, P76=0.15600376020932086, P77=0.24686139017370307, P78=0.31486998325760485, P79=0.46184431421142297, P80=0.35447911750983474, P81=0.17825190651741715, P82=0.1368595074041441, P83=0.22901441893066957, P84=0.36649665544474935, P85=0.28220258977317836, P86=0.14288859375030566, P87=0.19226907457463369, P88=0.6393834320311196, P89=0.5618197597149657, P90=0.3533290819299915, P91=0.1612987429877666, P92=0.06222382557499429, P93=0.025916635768106846, P94=0.03579664701273899, P95=0.06431168168431008, P96=0.15120984536515447, P97=0.5299879570406116, P98=0.3303263921393865, P99=0.049781799621427855, P100=0.028242832931440743, P101=0.017911171245818588, P102=0.014527129377535145, P103=0.03450605593002373, P104=0.041942378435869274, P105=0.027284538327361448, P106=0.019867538766246275, P107=0.06832889565447538, P108=0.07036928232374409, P109=0.05783077788126087, P110=0.042384272319668945, P111=0.052450040616171416, P112=0.05751479809198414, P113=0.02999823098083766, P114=0.02302392686848731, P115=0.02691689931917319, P116=0.0120497017672477, P117=0.004169233864477023, P118=0.011514173729016298, P119=0.005922605870010108, P120=0.003113424023977672, P121=0.006503675696095253, P122=0.0032581817669851145, P123=0.003092344767468267, P124=0.004420438964850171, P125=0.004163775415434008, P126=0.002552847339487353, P127=0.001494684985026686, P128=0.0020610693481630652, P129=0.0024720026246794415, P130=0.0017528703510498787, P131=0.0018460777848079836, P132=0.0022011805739695833, P133=0.0026526829317799322, P134=0.0009556074989319362, P135=0.0006185108564670154, P136=0.0011006594636658686, P137=0.0008466029261648038, P138=0.0015693776314408534, P139=0.0025258158642519177, P140=0.0032541347718092325, P141=0.00529859112118953, P142=0.0062858860179225255, P143=0.003817591617850786, P144=0.004596603520132103, P145=0.0027918279220208535, P146=0.0015292256291164397, P147=0.0006582283661792706, P148=0.0004840714473761823, P149=0.00044908204807370865, P150=0.0003987739276895844, P151=0.00034693365748361877, P152=0.00027527375338821496, P153=0.0003182219214894139, P154=0.0003032390457737834, P155=0.0003627345545264139, P156=0.00027108177305530093, P157=0.00041822987162171734, P158=0.00048769030544786317, P159=0.0008450646159213357, P160=0.000770044144835516, P161=0.0008729907137537336, P162=0.0012783390310937314, P163=0.0006318050417792901, P164=0.0004925416524388685, P165=0.0002871454146529533, P166=0.00023597404209070477, P167=0.0004706484212898753, P168=0.00043345604980444597, P169=0.00039490073949403334, P170=0.00014406600124843683, P171=0.00014370968979610321, P172=0.00016140441442178574, P173=0.00018719007481899567, P174=0.00010671547920396019, P175=8.668184188579691e-05, P176=9.501130845023187e-05, P177=0.00014181469423330003, P178=0.00011328823879539805, P179=1.0368527456254754e-05),Row( P0=-453.80403835501386, P1=32.4113586573327, P2=-11.103077259937011, P3=0.8309406425292943, P4=-6.0674302597591305, P5=-6.127441506003057, P6=-8.465151559132096, P7=-3.735160510056886, P8=-1.3812083390097052, P9=-3.2222937559033498, P10=-3.114953374882966, P11=5.326521692296337, P12=7.130366455020146, P13=7.792638756615035, P14=10.06026976973668, P15=7.303537113608561, P16=8.329954214847332, P17=7.013845101586489, P18=5.14363497213635, P19=8.968682643184367, P20=4.4995021594006985, P21=7.2371843499512, P22=3.662245387780322, P23=2.7373406603556183, P24=2.6055442573508065, P25=3.739655892806378, P26=3.4278111343871944, P27=5.70408765069001, P28=0.8297306007070696, P29=1.896013571291677, P30=0.9179873197314081, P31=3.2007177436731147, P32=0.7465658786732271, P33=2.6470925581924596, P34=1.0138255043523023, P35=2.390280925782521, P36=1.7031316651697195, P37=2.6134100939513694, P38=2.181913744886017, P39=1.7895944668839787, P40=0.632483606808466, P41=0.6035303876989454, P42=0.6439938151699602, P43=0.6725700346433823, P44=0.6761813622604245, P45=0.7236293767130078, P46=0.723246075593684, P47=0.6489991119026759, P48=0.6229501341933495, P49=0.7004273077007896, P50=0.6867404817886889, P51=0.6519445891741392, P52=3.397311561730781, P53=0.390918484954598, P54=0.009193332282718215, P55=0.0010783295403690484, P56=0.0007650267613353748, P57=0.0015420808942213846, P58=0.0025506636047110468, P59=0.0048873200580827015, P60=0.014941047876215826, P61=0.07602231849321328, P62=0.16552104505440937, P63=0.16250380096997538, P64=0.1571553135586699, P65=0.5167679143838342, P66=0.7345064909061787, P67=0.6040619499296901, P68=0.7179083760101177, P69=1.0844282810321717, P70=1.1656833282841355, P71=1.3615629474819324, P72=2.013393691203675, P73=1.7649168923477259, P74=1.2395968011983836, P75=1.1445747571197895, P76=0.9401439156900931, P77=1.8983816276148537, P78=3.393164417790345, P79=1.547648838059097, P80=0.15162227811494003, P81=0.1160758151173542, P82=0.15071734536109344, P83=0.09840982710967787, P84=0.06968719746818255, P85=0.06289684692405849, P86=0.0814403600095166, P87=0.15258320077685608, P88=0.4318785360854473, P89=0.41302799847554234, P90=0.5295690341596935, P91=0.6396589182185654, P92=0.8603961267736364, P93=0.6491352631000356, P94=0.8947817693974155, P95=0.7654973570436382, P96=0.3333880208951804, P97=0.5666066474260967, P98=2.2762424609935294, P99=10.211330572799387, P100=4.504965054590657, P101=0.13312683377173712, P102=0.10070136865128564, P103=0.15226795207824761, P104=0.14348111097830196, P105=0.08850839484542039, P106=0.0675229302285413, P107=0.08490984236768279, P108=0.24773777449980505, P109=0.4851534686649051, P110=0.2579699208475662, P111=0.1564322340110263, P112=0.13110583944505189, P113=0.05180660465037082, P114=0.04637343865490137, P115=0.057972957605770446, P116=0.03833471759911976, P117=0.02053258386937509, P118=0.012244127498348053, P119=0.022573054293647497, P120=0.025016937419284787, P121=0.01375628829629367, P122=0.012207496748676641, P123=0.008512010097623559, P124=0.008665950225464828, P125=0.013491366194221604, P126=0.012926580654908721, P127=0.005325132318860547, P128=0.0038603898994092594, P129=0.003986258663536361, P130=0.006393811563839422, P131=0.0048641688575468784, P132=0.0037431174573250353, P133=0.007344559548289138, P134=0.010501324108254875, P135=0.005528148990191172, P136=0.004914550896040097, P137=0.006351509408901929, P138=0.006288599525666654, P139=0.004109854313512712, P140=0.003518438027151494, P141=0.00618915915109716, P142=0.006014658520136685, P143=0.0052663670913773684, P144=0.018240776534318628, P145=0.020352824573252933, P146=0.008826768775251051, P147=0.005947901273314729, P148=0.018283319408513167, P149=0.010514999262342105, P150=0.0016370641082338675, P151=0.001050736432776085, P152=0.0009187487797201609, P153=0.0009997562727416174, P154=0.0011573465106887526, P155=0.0010290730742791576, P156=0.001318738849842233, P157=0.001578847822330569, P158=0.002103422495671074, P159=0.002832782687538105, P160=0.0019167677243817, P161=0.002484406985650618, P162=0.0022495209989483104, P163=0.001228979822612213, P164=0.0007230141865371306, P165=0.0009257478460303376, P166=0.0010416996735011935, P167=0.0010982629971273276, P168=0.0006979957737248383, P169=0.0005287910848939706, P170=0.0006541913083931201, P171=0.00029235844528702974, P172=0.0004328715740764307, P173=0.0005296577714886211, P174=0.0004407067026686442, P175=0.0004413569219779286, P176=0.0004960743086742053, P177=0.00045288834714551056, P178=0.00024281918375150324, P179=2.63597518615281e-05),Row( P0=-462.7176532026471, P1=28.722016277215257, P2=-11.068964374539203, P3=6.5098102067562404, P4=-2.3778699030939987, P5=-3.2343427681811114, P6=-0.8163439210000897, P7=2.191120352785074, P8=1.797735573811003, P9=3.2007827826654203, P10=0.8951447359714484, P11=8.016257830455997, P12=8.914969749482296, P13=8.357710932027706, P14=13.558338411140573, P15=9.92066366428928, P16=9.114916626049403, P17=9.94724097961154, P18=4.846330800604026, P19=8.891964084566517, P20=2.9233283574360223, P21=7.283440115421894, P22=7.895256289805892, P23=7.00669976836939, P24=4.167774961335319, P25=2.622921830554796, P26=1.8336387404162666, P27=5.820678243061872, P28=4.179335065962957, P29=5.042406702336646, P30=1.7716885044299089, P31=4.168315589369687, P32=2.125244174504822, P33=3.037354549827191, P34=1.3918426416469556, P35=2.251500640577899, P36=1.1812051527374448, P37=2.0278556524571294, P38=1.044016820625461, P39=2.2018394312899265, P40=0.6565416906753426, P41=0.6895830513542337, P42=0.7547350954589377, P43=0.7358218902648503, P44=0.7388976037542863, P45=0.7704231543693572, P46=0.782023803784096, P47=0.7582016454298627, P48=0.7515313486816535, P49=0.704894296651011, P50=0.6666903106643901, P51=0.6843818866186906, P52=4.872203612194704, P53=0.7014310749761287, P54=0.02245907784416975, P55=0.0024923013655684416, P56=0.0008100940024906809, P57=0.000675802714269884, P58=0.0007920867939362387, P59=0.0024472323883909814, P60=0.006825711398818171, P61=0.015981790106117805, P62=0.0663743276623645, P63=0.1622636753413045, P64=0.16953728601192586, P65=0.2164024000049174, P66=0.3695068866245315, P67=0.47608539631352453, P68=0.8697363414356944, P69=1.7920138584856977, P70=1.8837644053987697, P71=1.104581432218155, P72=1.3333720520638523, P73=1.5307557676814414, P74=1.9879767313780177, P75=2.4063533892225286, P76=1.4068011793951345, P77=0.5041572088624336, P78=0.14647409245961152, P79=0.11482241095215895, P80=0.1293057752776147, P81=0.3354195689183627, P82=1.0254495928830114, P83=2.9629902857937034, P84=4.137354600539705, P85=2.7400553303936688, P86=1.4816916277279037, P87=2.0035254224081798, P88=2.7509773434947125, P89=0.8776698202764422, P90=0.5275466500506761, P91=0.610306913519588, P92=0.6716589307129592, P93=0.4243463410051833, P94=0.5294998426590188, P95=0.9178572947309249, P96=1.3852123289234783, P97=2.69700534851631, P98=1.086562975956198, P99=0.45646039851014486, P100=0.4663199321886181, P101=0.1207768456634603, P102=0.12121655029397194, P103=0.6240569668526798, P104=0.4515846845129578, P105=0.19225250318219908, P106=0.22240922055775217, P107=0.4658765576169295, P108=0.8629169411115006, P109=1.6103568928892755, P110=1.0115854494187555, P111=0.06285773174591161, P112=0.07399052343131039, P113=0.05427408166178269, P114=0.13442530794027618, P115=0.21422488378975504, P116=0.07727769370571017, P117=0.1014239076623415, P118=0.10991651513517946, P119=0.10702139862017167, P120=0.11688777225900944, P121=0.04477122074926024, P122=0.023665670239840945, P123=0.03338008593773388, P124=0.07587958193742739, P125=0.0902431925128335, P126=0.023082481564303023, P127=0.015257832813156489, P128=0.010387556691326951, P129=0.018867463723762388, P130=0.03362452118078152, P131=0.0234487393135286, P132=0.025484649121875445, P133=0.031250011606912424, P134=0.02160830065416462, P135=0.027212090785979033, P136=0.016462914896649975, P137=0.016809479803344195, P138=0.010616171508182955, P139=0.02029858614217312, P140=0.03364499915648865, P141=0.023827820328663016, P142=0.022588411933908537, P143=0.031298090282984095, P144=0.019056045043268133, P145=0.014481275422561505, P146=0.011620920054926733, P147=0.00723292996293028, P148=0.004801114860688933, P149=0.003351604993775602, P150=0.0016191607769375886, P151=0.0014868384051023532, P152=0.00125397629495655, P153=0.0008045542646967753, P154=0.001352767013460923, P155=0.0011877516879104898, P156=0.0016615444326433255, P157=0.0031444351568778246, P158=0.0037144119673455027, P159=0.004373485789289204, P160=0.0031513320741767214, P161=0.0023555887825262734, P162=0.0023413694805254157, P163=0.0016201464591135018, P164=0.0011921438327906254, P165=0.001082414496035678, P166=0.001245122904973376, P167=0.0016840243327953558, P168=0.000999535237914881, P169=0.001159108544116526, P170=0.0012582982630103905, P171=0.0009478429116120912, P172=0.0007182675356563008, P173=0.0007413245619445663, P174=0.0010728023780104273, P175=0.0009209486649886891, P176=0.0008088309867319988, P177=0.0007141455664037828, P178=0.000502448835646886, P179=5.394722222803238e-05),Row( P0=-650.2233306464257, P1=74.2232116786221, P2=13.866311327260124, P3=18.400535735115408, P4=-0.04746798026923832, P5=-3.1129711131041278, P6=-1.4770223185627334, P7=1.7319382672385746, P8=4.905116504330026, P9=-3.6042607588108577, P10=-3.795359237139981, P11=4.6071876825994895, P12=-4.251134580661975, P13=0.7852593849134937, P14=2.198028142602311, P15=-6.5563997869450965, P16=-0.6536818599803949, P17=-3.0392129828644854, P18=-1.6142209908852116, P19=-0.4157161646056679, P20=-2.5563270537660956, P21=-2.0757424034099192, P22=-3.9195717134391894, P23=-0.8199916865707738, P24=-5.241640467145982, P25=1.1279713631285675, P26=-0.6950220063791958, P27=4.055690887062327, P28=0.7097212556161875, P29=4.276250033690537, P30=2.798663583469936, P31=5.739085220383501, P32=1.1462137546803748, P33=3.530469954821304, P34=-0.4541898895801479, P35=4.159064731396647, P36=2.573544402734241, P37=3.9744919359737003, P38=0.8670548906830113, P39=-0.15632245969136177, P40=0.7242182876750046, P41=0.7454038707181322, P42=0.7316784265465907, P43=0.6819048998629152, P44=0.6439875236201318, P45=0.628318052507128, P46=0.6295244457839045, P47=0.6177077729675801, P48=0.6573503895750993, P49=0.7213073402489839, P50=0.7190704305435344, P51=0.7043798847379469, P52=7.561057724779053e-05, P53=4.864279162268214e-05, P54=4.4224782835452985e-05, P55=6.689874720085355e-05, P56=0.00031427721158513734, P57=0.005998983914687044, P58=0.033106150247394425, P59=0.07983171285606036, P60=0.12621837984932752, P61=0.07609131484949366, P62=0.03724188317755122, P63=0.005092249762087328, P64=0.0015755973369870966, P65=0.012177235653956467, P66=0.025992797241409633, P67=0.028138996627003622, P68=0.03568583847644082, P69=0.04209279036983939, P70=0.02958260747720949, P71=0.011761946682817903, P72=0.010330437446035524, P73=0.010559626099657379, P74=0.011586423559686747, P75=0.01450711611616481, P76=0.0194121329886279, P77=0.048404518716198754, P78=0.05573333238090043, P79=0.047742660088907374, P80=0.021162556998250974, P81=0.01251348498123768, P82=0.012977267498338581, P83=0.011529275867204217, P84=0.006775659317044269, P85=0.0037628353283850145, P86=0.0061128818588461415, P87=0.03288289078392388, P88=0.05316764826141264, P89=0.02764003354951904, P90=0.019330493868622987, P91=0.014558067055041502, P92=0.004973558637570285, P93=0.009512384548202378, P94=0.05499094689896105, P95=0.026355229364394016, P96=0.008388006986773191, P97=0.002491641175673245, P98=0.001607503109896695, P99=0.003354165264699112, P100=0.004119262775365856, P101=0.0016075133115831316, P102=0.0012643671554149057, P103=0.000991449958030404, P104=0.005402947394130046, P105=0.0031660021347274156, P106=0.001244402901829083, P107=0.0013359635070949892, P108=0.0031117642909854674, P109=0.0017266891930587183, P110=0.0005560675746333269, P111=0.0005968949422540719, P112=0.0008358646033557092, P113=0.0005165670682484979, P114=0.00023178464767096286, P115=0.0007036463767989888, P116=0.00018156166683880938, P117=0.00014876471379298459, P118=0.0002547984791297376, P119=0.00011815039070166712, P120=7.456309074623373e-05, P121=0.00019662811007949436, P122=0.0002787142255721019, P123=7.68246727476266e-05, P124=4.209078397316543e-05, P125=4.395533442583276e-05, P126=6.878735592848816e-05, P127=7.882320728999984e-05, P128=8.422681969607075e-05, P129=4.541652038660263e-05, P130=6.0264421300346664e-05, P131=3.7379432711448117e-05, P132=4.4736008089712724e-05, P133=5.474811676801731e-05, P134=4.001561052546132e-05, P135=1.1356561324990959e-05, P136=1.398736880082358e-05, P137=3.1446767384501006e-05, P138=4.517966072403179e-05, P139=6.832096636747191e-05, P140=9.938733582785977e-05, P141=0.00016069642583852177, P142=0.0001359200498510547, P143=0.000125478258042777, P144=0.00012916576113901888, P145=0.00013488520245612926, P146=0.0001066371869569839, P147=7.997117793116809e-05, P148=6.506738888629566e-05, P149=6.168108789585527e-05, P150=7.208974493263929e-05, P151=9.490963396039588e-05, P152=8.731138369553589e-05, P153=8.974079760404561e-05, P154=8.746337416969056e-05, P155=8.795792945120263e-05, P156=0.00012202503828339928, P157=0.00011225476008542101, P158=8.964829001173302e-05, P159=9.950648953168376e-05, P160=9.812204013503251e-05, P161=7.69123333695918e-05, P162=6.073165281601117e-05, P163=5.388885683991818e-05, P164=4.949002467257299e-05, P165=5.781206028160984e-05, P166=3.8290395458150866e-05, P167=2.7271050521297534e-05, P168=2.835125233786622e-05, P169=3.720015496756457e-05, P170=2.6280557080178935e-05, P171=1.0402581032356385e-05, P172=7.887155261140234e-06, P173=1.3192581016784233e-05, P174=1.6363233744940232e-05, P175=2.047398503863049e-05, P176=2.6577722372013226e-05, P177=3.150649942090739e-05, P178=1.2382637591859666e-05, P179=1.643274632426431e-06),Row( P0=-458.34125667342187, P1=26.442816229241604, P2=-11.159797807575812, P3=6.935253546357897, P4=-2.6352506468741987, P5=-3.2583401855519116, P6=-4.250358121794719, P7=4.78557761901452, P8=3.921487298627834, P9=5.475683200353039, P10=2.4036093163048435, P11=8.386668342385837, P12=10.416479133998397, P13=5.843699919998906, P14=9.772890806317688, P15=5.245337962172622, P16=5.158927990952741, P17=8.631683747010719, P18=5.41091941450815, P19=11.933706710038516, P20=4.118447275710762, P21=7.894417475080985, P22=4.959649740310253, P23=4.6590795661284945, P24=5.645159745158152, P25=4.820916845083977, P26=2.4107867177525097, P27=4.153870192232634, P28=1.0321645982270993, P29=3.801875918560265, P30=0.46442245161467327, P31=2.9077770555516524, P32=2.764070618580795, P33=4.198926043912312, P34=0.22738224141726712, P35=1.4321302638309676, P36=1.6901703459524455, P37=2.8108231658830127, P38=-0.5268457078734369, P39=0.6189243493915897, P40=0.6609553880877953, P41=0.6770367601299558, P42=0.664592177250421, P43=0.7104003077541513, P44=0.7876991160471376, P45=0.84557111128258, P46=0.751971168255532, P47=0.6872590071484647, P48=0.7599060346710963, P49=0.7572865261366383, P50=0.7193716649250964, P51=0.6904151504369418, P52=1.883120952087082, P53=0.27488902828325645, P54=0.011584293840640978, P55=0.001644791760487146, P56=0.0006576419142977785, P57=0.00029426712884951967, P58=0.0003873478539259294, P59=0.00480107859353118, P60=0.02684536686647723, P61=0.0766241470778082, P62=0.14148909729576475, P63=0.18568978895966262, P64=0.16407419431481365, P65=0.6221046805217749, P66=0.8362701445909376, P67=0.5729369373921471, P68=0.30893109925385237, P69=0.25890901657081417, P70=0.46549636548605583, P71=0.587841183524743, P72=0.9936113891605907, P73=2.5946259187972256, P74=4.899963044378813, P75=3.058801025720613, P76=2.767561239166366, P77=1.7124381887755962, P78=0.6507120026435202, P79=0.8738494875628002, P80=1.1435677622310954, P81=1.0705542518495383, P82=1.000280129478292, P83=0.8972568231545293, P84=0.971557609266794, P85=0.4634854745580894, P86=0.170334244707788, P87=0.16011314302772722, P88=0.3183018181105726, P89=0.28716037082706886, P90=0.4584143992985111, P91=0.7264219387673537, P92=1.4324376728853865, P93=0.9204931276366437, P94=1.9983046183385933, P95=2.4302442184376254, P96=0.8506836920412097, P97=0.27751466459731033, P98=0.07100263767533935, P99=0.04103625021903946, P100=0.21831374844705587, P101=0.6249543519211206, P102=0.31113988928971703, P103=0.1623155605482471, P104=0.17856348279721695, P105=0.12720480223408379, P106=0.4080258621863982, P107=2.4679784431873757, P108=0.7872671573487139, P109=0.13287907608838256, P110=0.1262949991646938, P111=0.3721226498316968, P112=0.29515984919742483, P113=0.08547033610308491, P114=0.11921937898411798, P115=0.20454184084097032, P116=0.440851777762907, P117=0.11782737608037559, P118=0.09115914936072936, P119=0.06952676620584615, P120=0.12433362956499248, P121=0.11254058324463231, P122=0.09759663309957499, P123=0.05856088552673375, P124=0.1722018091798659, P125=0.8233467565476007, P126=0.1554102800290124, P127=0.016486457598571088, P128=0.04149452546353607, P129=0.03303074591099174, P130=0.03462289277659198, P131=0.04173331021422832, P132=0.05210332441071954, P133=0.06175903976058172, P134=0.021272346036661408, P135=0.08805164331094714, P136=0.06083985834856549, P137=0.026076254204957246, P138=0.018414819956193505, P139=0.028238686527218664, P140=0.024062869637697826, P141=0.015325278694865649, P142=0.011175794151653822, P143=0.024126673645722196, P144=0.022476193841445655, P145=0.017504069883793483, P146=0.013040398123540423, P147=0.007914958771800082, P148=0.006166263603685807, P149=0.005236536519924052, P150=0.004300617168144971, P151=0.0033231035114032565, P152=0.0030507497889231696, P153=0.002826457724717009, P154=0.003682086034131547, P155=0.0038917162801705256, P156=0.005641061896144509, P157=0.008728950334760484, P158=0.0095627937073153, P159=0.010680248261522396, P160=0.008590611059759402, P161=0.009190085620272143, P162=0.01119865898931905, P163=0.009227242507657153, P164=0.007538193196537439, P165=0.0044091557875395706, P166=0.003747782625547814, P167=0.004174391340169376, P168=0.005846794854483289, P169=0.005565308433629798, P170=0.003640602763488945, P171=0.002873164802325572, P172=0.003052846975923845, P173=0.0032348267919638934, P174=0.002979081568909045, P175=0.003255551969490774, P176=0.0033759473772704165, P177=0.002363173627373999, P178=0.0013644708779396886, P179=0.00018609958585184903),Row( P0=-645.1193671603805, P1=75.23805384366533, P2=6.883626218974803, P3=11.149451697840016, P4=1.208792886609596, P5=-2.34882970633578, P6=-1.775127227714804, P7=-1.2284926743237983, P8=8.493804026626549, P9=-1.8133506946521538, P10=-5.04784909431162, P11=4.952691802932732, P12=-4.529005647083639, P13=-0.8914340048114466, P14=0.5789441431417188, P15=-4.569204546362031, P16=-0.9313072643027246, P17=-1.5146489639992178, P18=-2.531557367525511, P19=-0.8287199419200609, P20=-2.1554324331347248, P21=-2.764607349202797, P22=-4.4529921616293, P23=-2.5574534238593225, P24=-4.047258552951441, P25=0.7067526666871308, P26=-1.8019458584926773, P27=4.363351039985644, P28=1.6703388782995299, P29=3.5637324643843833, P30=0.2175463883379119, P31=3.127730605219716, P32=0.953657506620026, P33=3.246779127673768, P34=-0.5500588663066713, P35=2.61044982069796, P36=1.9945540757060407, P37=3.3364682533777055, P38=0.024311374834093787, P39=1.1902393140732812, P40=0.6791627585386527, P41=0.6765891067240054, P42=0.7010609619786753, P43=0.7178053694121317, P44=0.6633218712652753, P45=0.6449598043871203, P46=0.6694866199967295, P47=0.6290702731530061, P48=0.6668524276424299, P49=0.7455575523410571, P50=0.7083510742035322, P51=0.6817312947533007, P52=3.9693485724223525e-05, P53=4.688485111625047e-05, P54=4.028553219922619e-05, P55=7.849112526757907e-05, P56=0.0008156296319578145, P57=0.011515211039107917, P58=0.04030883752371366, P59=0.10279080394979258, P60=0.1284645532450884, P61=0.08626766304657572, P62=0.02497092423839027, P63=0.003605389653383131, P64=0.0061801418011251125, P65=0.018753468160568058, P66=0.044895183861131685, P67=0.043199365741910474, P68=0.04526366215101463, P69=0.06819769374073088, P70=0.06364256028717498, P71=0.03159232936540028, P72=0.009019104226928086, P73=0.011617929003421141, P74=0.023287201414533564, P75=0.021092850129094773, P76=0.01470327802378879, P77=0.044256044734314534, P78=0.0716246145847646, P79=0.05787750163837737, P80=0.03300222579118334, P81=0.04222896092671435, P82=0.03555290961189942, P83=0.020823352378928926, P84=0.020833462190691824, P85=0.016233470548296696, P86=0.010748991429427611, P87=0.006463336006293181, P88=0.01172014065707133, P89=0.028118789009204708, P90=0.038591430074071155, P91=0.010538588387037768, P92=0.02193006550252494, P93=0.01841980366490464, P94=0.010571380836082729, P95=0.024522845921649685, P96=0.01132281293571102, P97=0.0015871823103426554, P98=0.0033979020997655317, P99=0.005244899576518609, P100=0.005201345912277737, P101=0.000940781599980565, P102=0.001644318881906407, P103=0.003589231139434611, P104=0.0028871313368216936, P105=0.0012853349410130055, P106=0.0013620738214030793, P107=0.004272350613167148, P108=0.003175442403799106, P109=0.0010283533960221023, P110=0.0012695585683089971, P111=0.002176673877919362, P112=0.000800919346860786, P113=0.0007914833886294115, P114=0.0018584920074956536, P115=0.0008528073877283867, P116=0.0004946405030428304, P117=0.00225909633633646, P118=0.0006565029229484597, P119=0.0003121073508506983, P120=0.0007336422652474131, P121=0.0008848363142759245, P122=0.0009765773951767418, P123=0.0003745645798224613, P124=0.00014474953314834712, P125=0.00019678313046214365, P126=0.0001367959546373013, P127=7.501400802351416e-05, P128=6.889075465003566e-05, P129=6.188574960855174e-05, P130=6.819624831910299e-05, P131=7.083551190066829e-05, P132=7.025618288859304e-05, P133=8.755550957474741e-05, P134=7.214447078153168e-05, P135=6.563051291870025e-05, P136=5.8047059564190957e-05, P137=3.5993441980112325e-05, P138=3.777699732910257e-05, P139=9.019812169412539e-05, P140=8.491051611170682e-05, P141=9.051795499261481e-05, P142=0.0001705903437423301, P143=0.00019336879973519755, P144=0.00016040621023541836, P145=0.00013552554217809626, P146=8.987974761457083e-05, P147=7.070970709901534e-05, P148=5.869168225815946e-05, P149=8.108822602191617e-05, P150=6.301921710448184e-05, P151=0.00011674283952536618, P152=0.00010551929382859374, P153=0.00010306824666977256, P154=9.66811772379206e-05, P155=0.00011231985638784269, P156=0.0001530299405675116, P157=0.00020756958129225546, P158=0.00015436117758368145, P159=0.00016207275239153037, P160=0.0001215749231169557, P161=0.00015236542118536334, P162=7.904862131440259e-05, P163=5.6901291966146236e-05, P164=6.39324553221716e-05, P165=2.554543362851834e-05, P166=4.508834286483257e-05, P167=5.819182509615232e-05, P168=6.0985730758003804e-05, P169=5.96015408930039e-05, P170=6.336206866130104e-05, P171=7.318340879184066e-05, P172=4.93655808651609e-05, P173=7.702812941594973e-05, P174=9.276286415027684e-05, P175=8.382027464750447e-05, P176=0.00013302214999481334, P177=0.00011543413336621412, P178=3.929089044553724e-05, P179=2.898272351179794e-06),Row( P0=-534.0303084107627, P1=60.36425455802696, P2=-14.586345539311225, P3=11.81060818106867, P4=-0.933969339442925, P5=-7.384904939493008, P6=-2.4998932993887166, P7=6.545898763204675, P8=14.731075393976875, P9=3.0615789571293073, P10=-5.066381734501899, P11=10.74274806741728, P12=3.829928935769178, P13=1.7865060691168675, P14=9.49724604983422, P15=1.4103347612070838, P16=5.6845947187651475, P17=5.578581565824277, P18=-1.0873963608656918, P19=5.902548555551524, P20=2.815772432750363, P21=5.701759157460165, P22=3.3872592565821957, P23=5.9970916246887755, P24=4.1931799395690215, P25=7.813497309501632, P26=7.096824946098696, P27=10.847889586361454, P28=7.754614805175009, P29=7.805797024954804, P30=3.2908485822831484, P31=5.562443907921027, P32=2.7899224105149325, P33=6.326598654178427, P34=-0.42851283948573904, P35=0.11603252859185811, P36=1.2093596161756177, P37=2.8884462778917577, P38=1.683266350852623, P39=0.42568017466950336, P40=0.6580535108968999, P41=0.6527472198571049, P42=0.6838592734020911, P43=0.6774786932198267, P44=0.6948146686213408, P45=0.7323868120707739, P46=0.7447611121764315, P47=0.6816154957937229, P48=0.7178855776613013, P49=0.7733090059606357, P50=0.715426587489707, P51=0.684538132514555, P52=0.15087522189916508, P53=0.005680198673660214, P54=0.0003405474920534449, P55=7.302882497219636e-05, P56=0.00011428606009867373, P57=0.0015826228437647808, P58=0.011350476780945033, P59=0.050836542306315124, P60=0.17491611434338444, P61=0.47581745727550057, P62=0.7025076540540892, P63=0.34104709513416814, P64=0.09575071894651369, P65=0.044916179112144385, P66=0.009711574174971009, P67=0.011801932826433928, P68=0.028194420921353994, P69=0.0711906652130392, P70=0.12268710482814331, P71=0.23790060228311855, P72=0.25859602764233375, P73=0.13902699881402952, P74=0.13963436843727262, P75=0.056708965213138694, P76=0.08151770902951345, P77=0.2729725134050726, P78=0.26681327696303186, P79=0.150699826155554, P80=0.0852045214993991, P81=0.2212387165041545, P82=0.5495372956718517, P83=0.32625780567695334, P84=0.22743114299210615, P85=0.3488878324716606, P86=0.26719278461748347, P87=0.26950176721925956, P88=0.4046308811744032, P89=0.20391098194098542, P90=0.2179124471492149, P91=0.2789680060023689, P92=0.16134973601151117, P93=0.1012159022587532, P94=0.17467906982892184, P95=0.08852061267224352, P96=0.11713489438702095, P97=0.33203454329024534, P98=0.291143268493158, P99=0.05215279788917684, P100=0.018435875871382488, P101=0.024825024128742157, P102=0.012672752467032655, P103=0.019704117648277047, P104=0.012114793122507074, P105=0.015368077601636483, P106=0.05514774931531401, P107=0.071045340681796, P108=0.09461835210091021, P109=0.030044018927837954, P110=0.009671962907110442, P111=0.08075630963679328, P112=0.12700547098216775, P113=0.05510834209972087, P114=0.020267381164137593, P115=0.10381416113166825, P116=0.05103269113490771, P117=0.010037957096913248, P118=0.028734983896758787, P119=0.021015614745125876, P120=0.02438668291220194, P121=0.019943022086160302, P122=0.013821256453819809, P123=0.010635731487090565, P124=0.005617792772325957, P125=0.0073837784662283525, P126=0.006775839404524737, P127=0.0046329256618954685, P128=0.005137557270745038, P129=0.008500266975628695, P130=0.007560007384690999, P131=0.009339940544434402, P132=0.006110582572417923, P133=0.004207471352068647, P134=0.002476824285572643, P135=0.0020649302185592397, P136=0.0014248520326422356, P137=0.001954174513606833, P138=0.003362462317258347, P139=0.004738169850899201, P140=0.007570993752077774, P141=0.010840124260068354, P142=0.011687863509852554, P143=0.00986958547214933, P144=0.008528935229755629, P145=0.005576863156973609, P146=0.003667098193129964, P147=0.0007547443124327472, P148=0.0007446822311266504, P149=0.000926417043578853, P150=0.0011531062320008943, P151=0.0009765330302338702, P152=0.0007599535242741964, P153=0.0005755505600709781, P154=0.0005257440303549782, P155=0.0004961055388658672, P156=0.0005465006921932479, P157=0.0004544989619096938, P158=0.000550999062539825, P159=0.000720180322433103, P160=0.0008687546417127348, P161=0.0011396666372451084, P162=0.0012737700135884162, P163=0.0008421252545835159, P164=0.0006943780344962007, P165=0.0007137985064907872, P166=0.00047929322813059687, P167=0.00033572934106656664, P168=0.000293394914075341, P169=0.00024585014685192427, P170=0.0002516640784219583, P171=0.00020258396191758006, P172=0.0001583284294374073, P173=0.00015694229701151974, P174=0.0001826119091010315, P175=0.00014715596237856318, P176=0.00011384996847517493, P177=0.0001169171059268708, P178=6.949876174318326e-05, P179=7.262401837282616e-06),Row( P0=-428.3102936951171, P1=29.37491842888047, P2=-6.817124899029449, P3=7.8399346092102915, P4=-1.3526921663514666, P5=-1.8291457315300435, P6=-2.7223008120001215, P7=0.8878997446897209, P8=3.145097069274053, P9=2.2255032705456723, P10=1.8049025130988512, P11=8.277382269082741, P12=8.829159986047738, P13=8.114576591673334, P14=11.798575713562908, P15=11.205791873904104, P16=9.576365543608809, P17=10.161957976908655, P18=4.789261468653964, P19=9.617454917393243, P20=5.531294506477256, P21=7.130513586615632, P22=4.469112796994002, P23=1.386219191656618, P24=2.990333077952493, P25=5.1613058015781395, P26=3.6174500708195074, P27=4.735037408518169, P28=1.4291047916785369, P29=3.157042665255608, P30=2.1262896328303795, P31=4.185718028179845, P32=0.8635023092918223, P33=2.030903884378784, P34=1.8578484323526812, P35=2.1381070368434143, P36=0.9551601106751528, P37=2.9563692511427733, P38=2.4634356796436654, P39=2.1834036726290442, P40=0.6500746880420095, P41=0.6595269830296895, P42=0.7092520805658726, P43=0.7240771131636485, P44=0.7639209158984074, P45=0.7278993168262362, P46=0.6767884599892888, P47=0.6807246886212612, P48=0.7512902521065205, P49=0.7366624132160444, P50=0.7427658280179562, P51=0.7079812598928393, P52=15.872221942774809, P53=3.496818600564717, P54=0.49802168087875265, P55=0.04005197123341126, P56=0.007245250751567461, P57=0.002041855544477339, P58=0.0008356443244485343, P59=0.002280343545140953, P60=0.006774749016770644, P61=0.016378536593415493, P62=0.07345938306433182, P63=0.171165523498344, P64=0.20425197631585085, P65=0.6507151519630012, P66=1.261038577567088, P67=1.0640344781669862, P68=0.9116950074811555, P69=1.0664441242817306, P70=1.7902929560593237, P71=2.757135372972331, P72=3.3155504305509425, P73=2.0187331207435713, P74=1.8795940958584834, P75=2.3568004804163363, P76=2.8939944338658723, P77=7.621189103742315, P78=11.987173725010736, P79=4.584778196176713, P80=0.3649813621565947, P81=0.28925801931655004, P82=0.20938137490009184, P83=0.16393144124758557, P84=0.3023135218884866, P85=0.38174797400439936, P86=0.30170219804981124, P87=0.30677091947571766, P88=0.722788559854157, P89=0.6404103738109098, P90=1.0440135807701876, P91=1.9914879716776401, P92=2.9637840861477356, P93=1.4690054542117053, P94=0.8424334269480049, P95=0.7792116069354578, P96=0.8781344666285562, P97=2.498876417689027, P98=6.799606304531279, P99=15.71696425472518, P100=3.976444915064363, P101=0.4102307262133452, P102=0.6959671841100458, P103=1.1289812761187583, P104=2.0243538937884242, P105=0.9725969961611284, P106=0.27145054014381464, P107=0.11052521278980325, P108=0.29424651346750624, P109=0.563331303569262, P110=0.49490126805108553, P111=0.4290804582514367, P112=0.33967470759564355, P113=0.3100804329963087, P114=0.30860987554776487, P115=0.14053121064487578, P116=0.06971643685569008, P117=0.10471455066969457, P118=0.0856712312952866, P119=0.06979118678890624, P120=0.0484257460928445, P121=0.02277120540869928, P122=0.02015516319167884, P123=0.03284449645568706, P124=0.033981504226994225, P125=0.04307321270245115, P126=0.02527821693255423, P127=0.0184313507514803, P128=0.016003912833021116, P129=0.013777114992265176, P130=0.006883685484793514, P131=0.008372729551172839, P132=0.03120631753499765, P133=0.08016165008363141, P134=0.0511148599466625, P135=0.027147248499350112, P136=0.02471526173497169, P137=0.04165784390744877, P138=0.03762447284000756, P139=0.01672258947954473, P140=0.021442432242909325, P141=0.040981019380317674, P142=0.03369599702512042, P143=0.01654553822067622, P144=0.013723339653947791, P145=0.017108637712745935, P146=0.012988750824434816, P147=0.008438605910425633, P148=0.008883192438453086, P149=0.00582520070000502, P150=0.0026838357136876913, P151=0.002550554835661103, P152=0.0020420095730330446, P153=0.0019399153418912802, P154=0.002924239570384949, P155=0.002355521149635808, P156=0.0036957088003360056, P157=0.008048442712599645, P158=0.008440120911717586, P159=0.011970646445441888, P160=0.005130579663562983, P161=0.004915983683696534, P162=0.002584454111893999, P163=0.001504664186191078, P164=0.0013126392358592126, P165=0.0012440700143848832, P166=0.0020081837545767125, P167=0.0031898947974863022, P168=0.0022745940990815724, P169=0.0017239839559369974, P170=0.0017811294524092993, P171=0.0016017404101223436, P172=0.0015320070157526379, P173=0.0016502031423201245, P174=0.002700940074298794, P175=0.002182062775570625, P176=0.0017454794872006963, P177=0.0014037093189366958, P178=0.000675824017450876, P179=5.305951493013506e-05),Row( P0=-626.6366930077238, P1=66.23320163974975, P2=1.543092456802819, P3=11.60442340537233, P4=0.26637433596153204, P5=-4.272638091358431, P6=-3.7457824142896423, P7=1.0593866566827512, P8=5.127976471525932, P9=-2.724630202673805, P10=-5.123906103393465, P11=-0.3059279561506984, P12=-3.583690832951339, P13=-2.2679096719996905, P14=0.4408287933603446, P15=-3.875763791626049, P16=-0.7564970948734355, P17=0.1778303305452105, P18=1.4828965985934837, P19=5.622297968060523, P20=4.811450001286639, P21=5.796800459123246, P22=1.7680192858493402, P23=3.5478738851590013, P24=-0.23004217364289947, P25=2.6241412455910496, P26=0.4429765528318462, P27=4.1500508338100435, P28=-0.3897710858607144, P29=1.0533974054534794, P30=-1.0089271191040923, P31=1.4091801244971864, P32=0.14904769171006507, P33=0.9926442102872268, P34=-1.8952350581045962, P35=1.6798262406614783, P36=0.7488021868371428, P37=2.320109235719856, P38=0.529401298514442, P39=1.044338335117297, P40=0.6404099808690242, P41=0.6333725152335054, P42=0.6406868537531799, P43=0.6547088443615614, P44=0.7017847124767104, P45=0.7076760416127607, P46=0.7500437616482643, P47=0.728624157444592, P48=0.7211358995729971, P49=0.7326433887727616, P50=0.7114334739525128, P51=0.6449630032221485, P52=0.0002979408305204574, P53=9.007357045291215e-05, P54=4.250448345513533e-05, P55=5.7002146655499655e-05, P56=0.00042649787463005454, P57=0.005595938015144989, P58=0.015798172404941, P59=0.03423920688545959, P60=0.04535917118550635, P61=0.060436705482137934, P62=0.16247345502428123, P63=0.24810612821019415, P64=0.21181828839061415, P65=0.15074788878527912, P66=0.04972980715789193, P67=0.012534831507059233, P68=0.011259574021895724, P69=0.009266250714652532, P70=0.007519929347620848, P71=0.00593209147448567, P72=0.0071645313928329945, P73=0.01343124182247433, P74=0.01811325072994736, P75=0.022927319973000727, P76=0.03100618133706257, P77=0.08573579606382023, P78=0.11168814810733693, P79=0.12070149774943244, P80=0.0476094738887864, P81=0.018567724994259606, P82=0.0068254922900130195, P83=0.007369197106729664, P84=0.01491380608130327, P85=0.01630916623797468, P86=0.01671945849376682, P87=0.014486935100930054, P88=0.01624417857274669, P89=0.012565946332215996, P90=0.027064028525593708, P91=0.13880165545978146, P92=0.10704400509016028, P93=0.016097334427723146, P94=0.010597903708104121, P95=0.005036521383142907, P96=0.0019245861865437523, P97=0.001779505969810511, P98=0.002190437135259962, P99=0.009410062998315517, P100=0.0659002581089761, P101=0.024536744998290803, P102=0.00273208579278937, P103=0.0016692502345328302, P104=0.0019227568161930139, P105=0.0020219031007782415, P106=0.005433189488494862, P107=0.03890175238822786, P108=0.010568174062312525, P109=0.0018132646433767345, P110=0.0012519850409131153, P111=0.0017070203017917006, P112=0.001708525233654989, P113=0.0037930954379003126, P114=0.0010725232514535777, P115=0.0004305691546502403, P116=0.0006275207753067261, P117=0.000944203669989548, P118=0.00048129459467217674, P119=0.00038317920566368344, P120=0.0004373516628418674, P121=0.00041879688620927586, P122=0.00043292812791854746, P123=0.0005435113181044016, P124=0.0007987766101916588, P125=0.00017528753889640316, P126=0.00026831468935756476, P127=0.00020838532619421729, P128=0.0002434952907630803, P129=0.00013271274495826004, P130=0.00013593472201259158, P131=8.63400169917012e-05, P132=7.764090362565582e-05, P133=8.047122051302372e-05, P134=0.00010208527119858172, P135=6.459673604704989e-05, P136=6.76405611946748e-05, P137=7.636610482054503e-05, P138=9.846662216067846e-05, P139=7.951300550583575e-05, P140=9.559823409755547e-05, P141=0.00012931650831481952, P142=0.00010127295235064613, P143=0.00010620961854936781, P144=0.00018427476168366563, P145=0.00017312053727770434, P146=0.00020122200561040174, P147=0.0001821893095941223, P148=7.15546892784248e-05, P149=7.38493666867508e-05, P150=6.199749469250985e-05, P151=5.3757499174560804e-05, P152=6.274319130192057e-05, P153=6.63264329387608e-05, P154=6.511948709263375e-05, P155=5.349741560797497e-05, P156=4.258480496274023e-05, P157=3.499021316454606e-05, P158=5.634297622729226e-05, P159=7.580837014521956e-05, P160=5.9282560082662356e-05, P161=0.0001407440633691627, P162=0.0001162989717229836, P163=5.997572138301081e-05, P164=5.66238456389099e-05, P165=4.351988856498333e-05, P166=2.8600784710493976e-05, P167=4.7926479556202496e-05, P168=3.8271531813942175e-05, P169=2.632536512237176e-05, P170=1.1249320672925244e-05, P171=4.334419642216488e-06, P172=4.800992579154083e-06, P173=1.1050306833405481e-05, P174=9.96486538121511e-06, P175=9.590641754417297e-06, P176=1.0918602874845773e-05, P177=9.200158593851647e-06, P178=3.4935822357190187e-06, P179=5.217037203573106e-07),Row( P0=-555.3301018514856, P1=40.159864682359824, P2=0.993996048143898, P3=3.017861052924557, P4=-3.0304681611798543, P5=-1.9990864950981133, P6=-2.439397820473181, P7=3.8855049634198857, P8=-0.3425079507563576, P9=-0.23837434889991516, P10=-2.7367710134259995, P11=4.442786621387728, P12=-2.0486275518129617, P13=5.899527666446382, P14=6.378357996706955, P15=4.350331234419753, P16=5.545469379198334, P17=3.359755102291649, P18=4.195385005921785, P19=3.6384300060360015, P20=1.5227686907891695, P21=2.727017313380394, P22=3.293159829325374, P23=3.2843287741040337, P24=1.056846196514114, P25=4.185056887489335, P26=0.3877586207019056, P27=3.079406315332765, P28=-0.9699045257281783, P29=1.517527080242631, P30=-0.35513218955374465, P31=1.356719975980035, P32=1.015192363157552, P33=1.437499972372893, P34=0.7925142811760284, P35=0.9706185576727886, P36=1.565787787484166, P37=2.1160216861550425, P38=0.8077049437917896, P39=0.6412451739756979, P40=0.7629512851347136, P41=0.7252422715678509, P42=0.7346120591351307, P43=0.7033238914576395, P44=0.6895197652375257, P45=0.6858193107465089, P46=0.6687863454853812, P47=0.6443539690359985, P48=0.629026688603417, P49=0.6886800369459217, P50=0.729484045197944, P51=0.7558451538400422, P52=1.2664507309784585, P53=0.12112409847988857, P54=0.002799310008099063, P55=0.0002544785378726676, P56=0.00011736623760590828, P57=0.00038325663208091404, P58=0.0017512939933891816, P59=0.005516052348067343, P60=0.012824458643753945, P61=0.031280815527014935, P62=0.10530073172261889, P63=0.131347903023641, P64=0.08093497700003345, P65=0.09601234915740438, P66=0.09227127499350532, P67=0.08171717492028578, P68=0.07118349378772632, P69=0.11407989467034738, P70=0.13243339323294473, P71=0.18688667638449133, P72=0.4367045952839745, P73=0.38609785469843283, P74=0.08101593898343959, P75=0.025339956521907, P76=0.011346052343956178, P77=0.011601590345922163, P78=0.01703914134540338, P79=0.017919109140149166, P80=0.011896980197920015, P81=0.022912782703543997, P82=0.06021421220537262, P83=0.20919837253395243, P84=0.387751072118544, P85=0.3575507471067939, P86=0.209211056992147, P87=0.12608009803356945, P88=0.15515369638399779, P89=0.08712182250263091, P90=0.08530580616191409, P91=0.16981763731341834, P92=0.4100760402003777, P93=0.1867206366984382, P94=0.028185174752886573, P95=0.04085199858589901, P96=0.091694366823546, P97=0.08164720575186746, P98=0.02554415269246558, P99=0.013107737887665996, P100=0.007936413677997148, P101=0.009485426134323767, P102=0.01156104111771382, P103=0.021440371998619142, P104=0.02611855548490604, P105=0.029593324151783992, P106=0.018819301572912735, P107=0.009549495360938798, P108=0.006693188182809035, P109=0.004033396264060674, P110=0.002393023132472623, P111=0.006362624549271869, P112=0.010517205750741635, P113=0.0034055645346988245, P114=0.0009509260930511665, P115=0.0011733428860593522, P116=0.0008460507623120968, P117=0.0008435803442268324, P118=0.0011020453784256717, P119=0.001526755796893028, P120=0.001961097456100769, P121=0.0027016962713336595, P122=0.001935422794122516, P123=0.0009759059767389708, P124=0.0009116133021806557, P125=0.0011745158578718377, P126=0.0014776719354014638, P127=0.0011364308966424843, P128=0.000973174429908502, P129=0.0009076546870583789, P130=0.0009509426424295067, P131=0.0007781002100879685, P132=0.0005811761036918347, P133=0.0010454709650350627, P134=0.000690641062416387, P135=0.0002387607578040187, P136=0.00028135026963818725, P137=0.0002118197721708859, P138=0.0001985588995452256, P139=0.00031180787605809997, P140=0.00035218697960750063, P141=0.00038963947752918433, P142=0.0005744326145973278, P143=0.0006734076414489967, P144=0.0011961636770536673, P145=0.0011110815619762083, P146=0.0010298859639606235, P147=0.001004367733915799, P148=0.0009320476038620977, P149=0.0008734379149136942, P150=0.0005890787260832316, P151=0.00043536042687691476, P152=0.0004906243835712485, P153=0.0003381487958330404, P154=0.0005031058708563445, P155=0.000502176144461901, P156=0.0004080430455413014, P157=0.0003352922886106241, P158=0.0003276031415693766, P159=0.0003444994217937716, P160=0.0004394017137341476, P161=0.0008953065827673025, P162=0.0008779459915454893, P163=0.0008393106779595845, P164=0.001044356684007734, P165=0.0016200412803768155, P166=0.0012682529508407923, P167=0.0019852276004173387, P168=0.0016667239089086698, P169=0.0007475799912287882, P170=0.0005724284509935054, P171=0.00031883598961920554, P172=0.00017165077643839613, P173=0.0002843234220826155, P174=0.0001402744021909313, P175=0.00012566674817507232, P176=0.00010983286555118934, P177=0.00012415936142116582, P178=6.656368520095521e-05, P179=7.830704629234739e-06),Row( P0=-524.7846923072831, P1=40.80378658827104, P2=-7.5704270661100646, P3=-0.5703987780021658, P4=-7.440201523099839, P5=-3.0844236247667975, P6=-4.170574376873345, P7=-1.4063296072652456, P8=-2.3483149720329317, P9=-2.695367043737016, P10=-4.384233748599902, P11=3.3715178948632514, P12=6.7823793282846765, P13=7.678313872398643, P14=11.844599256564486, P15=10.154379786012294, P16=10.84299901143391, P17=7.291153555492357, P18=5.780284898578365, P19=9.175518099897284, P20=3.829993450484109, P21=8.485517300245322, P22=3.772551314189368, P23=3.5884560430695633, P24=3.720257738141121, P25=4.5927561193335835, P26=1.8863581723137635, P27=3.8664570652811645, P28=-0.240124718798398, P29=1.556095080293419, P30=0.7832034718432285, P31=3.9660215677139914, P32=2.042482362244311, P33=2.7349012977442553, P34=0.688109000772589, P35=1.230484217388776, P36=1.9009438562494791, P37=1.6873268187642625, P38=0.5657387942380131, P39=1.3827891357849347, P40=0.6278515359911072, P41=0.5788365442130902, P42=0.5995975980281183, P43=0.6435771729527311, P44=0.6613465236119324, P45=0.6749489775296587, P46=0.702030128129246, P47=0.6498747064058771, P48=0.5989558484611771, P49=0.6432766122620398, P50=0.6570086349817916, P51=0.6616753296846095, P52=1.3808140574694243, P53=0.38865538298408897, P54=0.02623857857094132, P55=0.0017298439824590876, P56=0.0013034404959224412, P57=0.000592622575993977, P58=0.00026695220684772855, P59=0.0008733542921898592, P60=0.003683066633428117, P61=0.012051947204520096, P62=0.03917699729857546, P63=0.09174035728025598, P64=0.22067510470543547, P65=0.5766834307945513, P66=0.8904884941216461, P67=1.108788327706568, P68=0.6897430999253458, P69=0.4702811108263693, P70=0.7972443674969318, P71=1.3480412851840218, P72=1.3403704716246423, P73=1.1693610637779575, P74=1.288060882668673, P75=0.9287049010987645, P76=0.14084302017198821, P77=0.025188885022378094, P78=0.05457308032394089, P79=0.06499234016520829, P80=0.05852621184165256, P81=0.04913346471124503, P82=0.06484521018395221, P83=0.1728909904722424, P84=0.3459036200861824, P85=0.22779611648040357, P86=0.13239327659996422, P87=0.08988017652111269, P88=0.21151627753998645, P89=0.24032540456017476, P90=0.4285425498630999, P91=0.6639242439915873, P92=1.0690049617492448, P93=1.046072396983982, P94=1.3138966698021328, P95=1.648867851607418, P96=0.29520381662746886, P97=0.055491177208516734, P98=0.15736623804274927, P99=0.13514125908766475, P100=0.08880878526355075, P101=0.09023259924450583, P102=0.04754653729531842, P103=0.052598644400630126, P104=0.06416972324709763, P105=0.04633491592734079, P106=0.06631318538393985, P107=0.2315874994453752, P108=0.2072246508792566, P109=0.019418052093430914, P110=0.00890197543232641, P111=0.022911377886614504, P112=0.036585870265278195, P113=0.03798909328488131, P114=0.025366355923975133, P115=0.00701255289895687, P116=0.007180240263576458, P117=0.012721982936424573, P118=0.01055035878001919, P119=0.0051274854951741215, P120=0.004838508761749127, P121=0.004894820195557666, P122=0.003654932468972852, P123=0.003449852496428361, P124=0.010295468537142059, P125=0.006437672096064275, P126=0.0027188439946386742, P127=0.002080819073337841, P128=0.0014383834163651096, P129=0.0023015238452816613, P130=0.001407723504388972, P131=0.000655043108584641, P132=0.0012361059194368133, P133=0.0026171920278887616, P134=0.0016190696385651713, P135=0.002998635957179732, P136=0.003489063261139585, P137=0.00237631710710165, P138=0.0014464010876449847, P139=0.0012763335441410778, P140=0.0009229682746913308, P141=0.0010725921298420683, P142=0.0010316624851894648, P143=0.0006984123282068766, P144=0.0016495190133287896, P145=0.0018919498376732769, P146=0.0019100368618686734, P147=0.002834956139755743, P148=0.0015129632518401607, P149=0.0009632784270187547, P150=0.0007083843645591681, P151=0.0005635326556486908, P152=0.00036433507816269005, P153=0.00039861631543400397, P154=0.00033868001256081155, P155=0.00037257676520320555, P156=0.000552427188678224, P157=0.0007726490813657595, P158=0.0006675101943780745, P159=0.0005614158382267514, P160=0.000595537023587415, P161=0.0006628050823353705, P162=0.0005390162869828422, P163=0.00039531123784155236, P164=0.00032841692893178755, P165=0.0002894882463070462, P166=0.0003425732592605684, P167=0.00020307890876478107, P168=0.00011499127452229456, P169=0.00017649392258307914, P170=0.00015055378924423187, P171=9.908124356931243e-05, P172=0.00012115978447629966, P173=0.00014408881343429335, P174=0.00021819378395032562, P175=0.00017127294630410892, P176=0.0001426164211534571, P177=0.00011721184958024803, P178=6.59097738918689e-05, P179=9.28894445742918e-06),Row( P0=-597.9222339809662, P1=58.55707056672707, P2=-4.5515836436819015, P3=13.282827734324824, P4=-1.5727758043339128, P5=-5.233656129240292, P6=-3.1623546219806684, P7=1.7878282933845964, P8=5.029226279124782, P9=-4.131972920032498, P10=-7.479735422308716, P11=2.1040257939026663, P12=-4.118585740580614, P13=-4.066622452194445, P14=4.914294762458835, P15=-0.7069564999937475, P16=2.7279244079177567, P17=2.329336208529153, P18=2.276797530175359, P19=6.691729724538368, P20=2.012996938006988, P21=3.2403554508904966, P22=2.3581314132130298, P23=4.237747288961007, P24=1.8874410682627023, P25=4.995648656934072, P26=2.4946180380399516, P27=2.3615745181198866, P28=-0.658367384137745, P29=1.9090917300262364, P30=-0.2496652318614145, P31=1.354972348125068, P32=-2.0919349725952916, P33=-0.16579311729729973, P34=-2.753577752786131, P35=0.15016900163229885, P36=0.22492585672073545, P37=-1.1744774025550146, P38=-2.7685576451267093, P39=-1.5130919129161864, P40=0.604338758248055, P41=0.5672095367500616, P42=0.5667185507259767, P43=0.5883078852928807, P44=0.5891959804681673, P45=0.6295319689782813, P46=0.6713043082018672, P47=0.6790549858096623, P48=0.6236135512423361, P49=0.6990925928446269, P50=0.7165954711350363, P51=0.6425823710370825, P52=0.00036281546476069526, P53=8.94797063657463e-05, P54=0.00010291513090311728, P55=0.0002636326400517887, P56=0.0007268603014185687, P57=0.0029373314062901834, P58=0.002562667818721931, P59=0.007582266912860172, P60=0.051611776250876734, P61=0.1339400997906817, P62=0.2999407094651464, P63=0.33740645416662696, P64=0.1302169067029596, P65=0.1665847404790092, P66=0.20503327934603946, P67=0.11650236323573507, P68=0.04584928964081518, P69=0.011804043522620284, P70=0.02475063952355736, P71=0.01623312305560454, P72=0.02150302715809655, P73=0.028145966569655994, P74=0.03303811258581873, P75=0.030582623011136418, P76=0.01388170424976136, P77=0.012612080222557128, P78=0.02773666898012017, P79=0.043793458883705, P80=0.03534410448830146, P81=0.045295191244568545, P82=0.08454615988916551, P83=0.17002793144129053, P84=0.3772799414544174, P85=0.22502631621609143, P86=0.015825148232266568, P87=0.01492692716436937, P88=0.03641287412914571, P89=0.025291257832374868, P90=0.028434273463411486, P91=0.11205937122809498, P92=0.1334510385661077, P93=0.06319812965575593, P94=0.03577797562233918, P95=0.027187691612427515, P96=0.00975859313467295, P97=0.014729443468744588, P98=0.02940158295863804, P99=0.04208465009783433, P100=0.02935544258657078, P101=0.006597929372495991, P102=0.0009711683142688017, P103=0.006661178357223601, P104=0.023674474207116818, P105=0.0338279848375832, P106=0.019031472904509454, P107=0.001334758060207633, P108=0.028831423869069263, P109=0.04485570234995825, P110=0.0067222884850549605, P111=0.0011418646445896514, P112=0.0027189738269940756, P113=0.005732615485062027, P114=0.0011260638228423026, P115=0.0006671488183281988, P116=0.003000611058735233, P117=0.0010364187779195965, P118=0.00044758138560526377, P119=0.0006620781305478454, P120=0.000695631390489, P121=0.0005160021161307876, P122=0.0006042777600166712, P123=0.0007698243574146365, P124=0.0010735444322022742, P125=0.0001840791430638771, P126=0.00012391378419186104, P127=0.00021636367748598993, P128=0.00024594047223907314, P129=0.00018403884631601274, P130=0.00028409988106022863, P131=0.0005323300633003559, P132=0.00043342730276455584, P133=0.00010431804785571051, P134=0.00014176222404743686, P135=7.658600802995802e-05, P136=8.823506940675001e-05, P137=0.0002202105998800696, P138=0.0003443405474563679, P139=0.00029593307250041145, P140=0.0005450444016302053, P141=0.0008746310840229695, P142=0.000846580947361497, P143=0.0008502427746328099, P144=0.0005673026409042863, P145=0.0006075587235457451, P146=0.0005539712486901516, P147=0.0001191576092308246, P148=7.112221564112625e-05, P149=7.813773142976176e-05, P150=5.035568287495796e-05, P151=3.690937010299376e-05, P152=6.229745487117597e-05, P153=5.770058299999053e-05, P154=4.403118100647517e-05, P155=3.298085326845024e-05, P156=3.8880402284512146e-05, P157=5.358683476266275e-05, P158=0.00011988880896175784, P159=0.00024394889237928826, P160=0.00011918363708449662, P161=0.00010989342283931979, P162=0.00011183795393301051, P163=8.570739138419893e-05, P164=4.5631100884545534e-05, P165=3.8489800890829706e-05, P166=5.831566555408118e-05, P167=3.8349499315957506e-05, P168=1.655843307399425e-05, P169=1.1471514085350508e-05, P170=7.024589538614422e-06, P171=4.6000441869982935e-06, P172=5.187387776748342e-06, P173=8.14805119390363e-06, P174=1.029161765869761e-05, P175=1.1610249227698086e-05, P176=1.2240189547267923e-05, P177=9.676362676886597e-06, P178=6.3188436608714715e-06, P179=7.879645435883601e-07),Row( P0=-598.8121500048668, P1=55.87358842468784, P2=-5.578096970817802, P3=10.340489393021295, P4=-3.8880853868529486, P5=-7.405477383523068, P6=-1.9879046217086591, P7=-2.4252328166039407, P8=5.995803577796602, P9=-2.5808831092614586, P10=-7.925488534294718, P11=0.4771690645653458, P12=-3.9449166961463167, P13=-2.3363130493725777, P14=3.3187573856447736, P15=-3.2091390836149687, P16=2.7023423434067575, P17=1.9002858469627555, P18=1.1010800035319945, P19=2.650070039179425, P20=2.5239241096469, P21=5.52261755241711, P22=2.7124449720346204, P23=5.397378691874485, P24=2.428527974900611, P25=5.263401630687359, P26=-0.02416220196697992, P27=1.6741465006308363, P28=-0.7342825084464258, P29=0.6008600877724067, P30=1.0320691393537167, P31=2.2343641070751405, P32=0.5228017052615814, P33=2.9905322535001413, P34=-1.5978189861341743, P35=2.7001235924231173, P36=1.775513704143647, P37=2.0616757928461964, P38=0.5092605368562427, P39=1.6795388165641976, P40=0.7100237667449028, P41=0.6851887904712106, P42=0.6865661670270101, P43=0.7021428713151028, P44=0.7183235788101315, P45=0.7483623155335306, P46=0.7770730282515834, P47=0.745713770687237, P48=0.6883906845750207, P49=0.7030051086183345, P50=0.7089468895101909, P51=0.7100389503954893, P52=0.00048251918313819384, P53=0.0002143176110563917, P54=9.576621183477939e-05, P55=0.0001020953782958945, P56=0.0007307642668555363, P57=0.008872666344465903, P58=0.01964846533076845, P59=0.02752649274022203, P60=0.05549896407820755, P61=0.11433278281378577, P62=0.2736160817503653, P63=0.3052186984604725, P64=0.18289747807365478, P65=0.18315612692665595, P66=0.2911899547672925, P67=0.1784558085292888, P68=0.1023888424057907, P69=0.03808234257242351, P70=0.042158063357944156, P71=0.028530450955997277, P72=0.03291446691824236, P73=0.045516777252673994, P74=0.04998748162646231, P75=0.048612668869757424, P76=0.039357259181708806, P77=0.03735090167598801, P78=0.03357891131114694, P79=0.04123323943248343, P80=0.06536848931245838, P81=0.11154826907607704, P82=0.11333780996270476, P83=0.20022522221863526, P84=0.2766823930817497, P85=0.17995332842748593, P86=0.129307603594001, P87=0.1483238010371017, P88=0.19582733980034903, P89=0.12065677807633032, P90=0.09680234974446393, P91=0.13221318493245596, P92=0.27519819989549915, P93=0.16280419753016187, P94=0.15198546284491574, P95=0.12189438079650428, P96=0.04286254866375445, P97=0.022180044832609623, P98=0.05493177323904881, P99=0.07323422103143748, P100=0.03798003184011508, P101=0.031417400003430254, P102=0.018968522865499597, P103=0.008902900308406372, P104=0.01550465813744708, P105=0.011508760683194978, P106=0.013466780788216589, P107=0.030267040757329788, P108=0.030725838429929393, P109=0.027516623311739406, P110=0.012504667789298266, P111=0.017843103419780054, P112=0.02936592920679398, P113=0.023741069300442876, P114=0.02989927402287525, P115=0.027722774482330605, P116=0.015212008801198263, P117=0.02231835861637278, P118=0.00797224056451151, P119=0.014649781267307009, P120=0.003818797184648755, P121=0.006068728114401572, P122=0.006430221436077268, P123=0.00330567254804735, P124=0.0024567485959866866, P125=0.0018598862647645005, P126=0.0011344264738340167, P127=0.0013251148958847206, P128=0.0011356984430993015, P129=0.0009756707595427431, P130=0.0017437127210745312, P131=0.003115150213187408, P132=0.002391756977096343, P133=0.0014179977431546024, P134=0.0007655499463868475, P135=0.00031421205978696874, P136=0.00038885764457762093, P137=0.0004857815371495988, P138=0.0005300957530626922, P139=0.0009380978091735525, P140=0.0017429766254287956, P141=0.0021522586186877974, P142=0.0036443378750645843, P143=0.002773337094218535, P144=0.0016734173778493444, P145=0.001835709780873878, P146=0.001608044122489295, P147=0.0009213522907668089, P148=0.0004857036260534632, P149=0.0004517738479837909, P150=0.0003193153797123639, P151=0.0005021895735200929, P152=0.0006197184282835721, P153=0.0009991970169565098, P154=0.000860315161397543, P155=0.0006122140319347335, P156=0.0003984229739235889, P157=0.0005916374005298819, P158=0.0007898025619137417, P159=0.0007289443501003603, P160=0.0005441698347288718, P161=0.0005543086841147482, P162=0.000772232634051626, P163=0.0006682406488805586, P164=0.0004682168177080553, P165=0.0003800048589359943, P166=0.0004444944487600767, P167=0.0005394721114163661, P168=0.0007421832550834568, P169=0.0005428439234703084, P170=0.00025493982928604823, P171=0.0001133475436222905, P172=9.23108175298049e-05, P173=0.00015438779458529852, P174=8.90095515434331e-05, P175=9.72806020683532e-05, P176=0.00012435936722517837, P177=0.00021424190410021282, P178=0.00011563729661935712, P179=1.5839124591056925e-05),Row( P0=-741.1410183476543, P1=20.074039066719205, P2=-18.625213029204158, P3=-9.905016047947056, P4=-19.944547290636116, P5=-13.307391056318458, P6=-5.0236214947592375, P7=-5.977935201659153, P8=-3.87392284601038, P9=-10.30421082280654, P10=0.0851739801110259, P11=-1.230667007501873, P12=1.3307123760500181, P13=8.959453517961666, P14=4.744571018880369, P15=9.072947289432136, P16=2.0349029337180427, P17=0.26957068362116765, P18=-0.44911875796325956, P19=-3.23973663836987, P20=-0.37333474780387155, P21=1.0993280274015214, P22=-0.2327510528574659, P23=0.04604531160928148, P24=-1.5682803152015554, P25=-1.6843782610515965, P26=-1.2840484464430368, P27=1.541548009358005, P28=1.1815782676531816, P29=2.24149611090887, P30=0.06041632152110927, P31=0.5697129915711184, P32=0.9352914985576103, P33=1.7179927467711642, P34=-1.7507364176955367, P35=-0.353467961867903, P36=-1.8171545294796927, P37=1.420613369093213, P38=-0.9413854904781417, P39=0.5549936615845628, P40=0.49965300478966157, P41=0.5634526351170703, P42=0.6340450721294065, P43=0.5838656601466805, P44=0.5585959680698444, P45=0.5167115212864613, P46=0.47436037348835314, P47=0.4564641519860928, P48=0.4578814846024253, P49=0.4602905381859953, P50=0.44641562589864253, P51=0.46783660271976624, P52=2.2437546509317896e-07, P53=3.8489630664831823e-07, P54=1.8963353949645468e-07, P55=1.222386542985154e-07, P56=5.270531365463649e-07, P57=1.953617621006794e-06, P58=1.8041202089425125e-06, P59=8.763328958507921e-07, P60=6.896170325481504e-07, P61=2.1553631478413015e-06, P62=1.0797388970685981e-05, P63=8.436883146227463e-05, P64=0.00017868580478689624, P65=0.00030671215512284426, P66=0.0009458402149866396, P67=0.0022224679950566782, P68=0.012126510925949475, P69=0.04315620416189634, P70=0.04633635486310656, P71=0.018979135481687795, P72=0.012375170150414415, P73=0.013618014455114057, P74=0.008159501979895265, P75=0.004824596401785468, P76=0.0012037727744919457, P77=0.0006365802788185506, P78=0.0006929530084442673, P79=0.0005709563077431328, P80=0.0004842588453367716, P81=0.0005295323474677382, P82=0.0006708545452312709, P83=0.0010155503295201716, P84=0.002763557327820593, P85=0.010541932923666817, P86=0.014692230214320855, P87=0.00930381568247875, P88=0.009819407401237507, P89=0.010655833941395267, P90=0.009866643210735718, P91=0.006422446639611269, P92=0.006637234153077423, P93=0.004580241046929509, P94=0.0025332659196831892, P95=0.0032737612997477896, P96=0.0010358963658205623, P97=0.0011468270584678866, P98=0.007904374928973498, P99=0.003714735779778481, P100=0.0002724550386758299, P101=0.0002543960488057947, P102=0.00026640922110268407, P103=0.00015983963576943192, P104=0.0003045160311114912, P105=0.0018276365140088226, P106=0.0019575866794702287, P107=0.0009605523781348296, P108=0.0004993675745305863, P109=0.00016035733360201008, P110=0.0001080827297142801, P111=0.00013587276486718705, P112=9.517284944039547e-05, P113=0.00014719310802275968, P114=0.0004230174792145035, P115=0.00027077097619031514, P116=0.0001830568422367554, P117=8.870028519880877e-05, P118=0.00010810296107459965, P119=0.00011752811266308661, P120=0.00021500400640102823, P121=8.446509121118765e-05, P122=2.5707868816299835e-05, P123=5.370245263075991e-05, P124=0.00036842339191994983, P125=0.00023095851678811677, P126=4.096190689848006e-05, P127=2.0750147538028415e-05, P128=6.34993951904646e-05, P129=0.00012306512637921997, P130=2.440456245209462e-05, P131=2.0076955140752723e-05, P132=2.321267442094351e-05, P133=3.0350670415101257e-05, P134=1.2752073011214995e-05, P135=5.080293141230246e-05, P136=5.4426592939288106e-05, P137=4.321380503428933e-05, P138=6.975351177355613e-05, P139=4.081764276807407e-05, P140=2.9088493058806528e-05, P141=2.436050287467844e-05, P142=2.9249424584264567e-05, P143=1.254434477310546e-05, P144=1.362854868969635e-05, P145=2.588152289662776e-05, P146=1.5624960546073526e-05, P147=5.267880519191913e-05, P148=5.99408008028349e-05, P149=2.5439353310327604e-05, P150=4.181474647712565e-05, P151=2.0351564872205016e-05, P152=3.309172175335367e-05, P153=1.575366327245238e-05, P154=1.4709521070997068e-05, P155=1.3077165776399215e-05, P156=6.475528638861582e-06, P157=5.956119970775993e-06, P158=6.191649964205455e-06, P159=5.277952569915743e-06, P160=2.0671843801618993e-06, P161=1.357456699321749e-06, P162=1.5602697901223512e-06, P163=1.553339829268495e-06, P164=1.717315012255885e-06, P165=1.4458198427537043e-06, P166=2.5008089624085053e-06, P167=1.8413747259190286e-06, P168=2.357489358865184e-06, P169=5.321306058536254e-06, P170=1.1164977371856037e-05, P171=7.452519044812588e-06, P172=4.333813739215848e-06, P173=3.6969803234004948e-06, P174=3.392111303859715e-06, P175=3.875755745993242e-06, P176=9.022884163101472e-06, P177=1.0844679921175548e-05, P178=6.528339477068213e-06, P179=6.64939648245211e-07),Row( P0=-666.0304917028758, P1=24.667265148985457, P2=-26.841463504223157, P3=-9.171130606268141, P4=-21.344803928854542, P5=-13.20964190445155, P6=-8.515863706249457, P7=-10.395856466919804, P8=-2.7822299467629583, P9=-11.121050635737726, P10=-2.863797426196292, P11=-2.2031191368732492, P12=4.676217882010578, P13=6.01724770816233, P14=2.0183487638899384, P15=5.845265140964426, P16=-6.652052677486084, P17=3.0825748996460325, P18=-0.6625587098999773, P19=2.478356289395389, P20=1.3762578691287781, P21=-0.746850461622234, P22=-1.9087550603967478, P23=-0.10137549147524849, P24=1.249785145027643, P25=-0.04412203826717871, P26=-1.0534706046507578, P27=-1.314700007206063, P28=-2.7938243944884, P29=0.42819711731269355, P30=0.8240625205309895, P31=-0.5377788165302271, P32=-0.6024708632796381, P33=0.26358500985265065, P34=0.362152052208805, P35=-0.5264425516662031, P36=-2.1761895669701405, P37=0.4128223713348419, P38=-0.4973558972247087, P39=0.40145565804589944, P40=0.45485787102650094, P41=0.4460151627131454, P42=0.42699636932918283, P43=0.46427605823816853, P44=0.5060528542917286, P45=0.4716282077007106, P46=0.47625051539881724, P47=0.45345488294761455, P48=0.4227484498325055, P49=0.410674388320085, P50=0.3956033495071156, P51=0.41553960057995337, P52=1.6658041209574082e-06, P53=1.733023755573751e-06, P54=1.416907366990761e-06, P55=1.3773156570249592e-06, P56=4.112491118952045e-06, P57=5.894958222521859e-06, P58=2.899555682587995e-06, P59=2.738829375717871e-06, P60=4.336672361856105e-06, P61=1.0673555611096257e-05, P62=0.0003027492939000199, P63=0.0014374623140677714, P64=0.0022597037695689477, P65=0.006528993210934525, P66=0.016720014473889072, P67=0.025051213724477512, P68=0.019024448919426448, P69=0.022814141594919693, P70=0.04758328835713595, P71=0.04671634304213114, P72=0.024021247006136504, P73=0.026871111324646663, P74=0.03144466702908333, P75=0.03120520283463658, P76=0.010911965281301433, P77=0.013082693535055099, P78=0.019802874695551434, P79=0.02655226659970803, P80=0.015300152571243498, P81=0.008025833383817845, P82=0.01275905368097149, P83=0.045294342261339215, P84=0.06434329267210216, P85=0.03890993419859727, P86=0.013134074386441388, P87=0.009283069746596043, P88=0.011598113520073543, P89=0.016638961345428332, P90=0.033596427475169326, P91=0.07353316046520716, P92=0.0294846525831964, P93=0.01599445967345274, P94=0.011948122114090928, P95=0.016712640086876153, P96=0.006843929997623482, P97=0.015215331436172281, P98=0.026237130129814913, P99=0.02108799159399625, P100=0.01991820629603412, P101=0.00645022432189552, P102=0.0028515727723298174, P103=0.04606278554721799, P104=0.03627855542032763, P105=0.01687350547771769, P106=0.010950502000897127, P107=0.005994804485616123, P108=0.0047148232544624346, P109=0.0037474106112445004, P110=0.003606399701554232, P111=0.008846519329221226, P112=0.0379260010852817, P113=0.0106193786562596, P114=0.007671252019146309, P115=0.005043818094428715, P116=0.0015051482871880745, P117=0.0016294122474790572, P118=0.0010333371567126134, P119=0.0035313149654390965, P120=0.003108971185158263, P121=0.0016014131153897968, P122=0.0009071773634494353, P123=0.00047844650519423204, P124=0.0006631118722721953, P125=0.0008483729393013256, P126=0.0004589616299065555, P127=0.0008720215684306085, P128=0.002120137318845303, P129=0.000778651187221559, P130=0.0001525361444476145, P131=0.0004409026546591284, P132=0.0004629670088992663, P133=0.00019201215505858552, P134=0.00020890897819494052, P135=0.0002732864195177879, P136=0.00022774616499588287, P137=0.00035693254289755993, P138=0.00041056951337308157, P139=0.0005083498723006758, P140=0.0006198839496279492, P141=0.00045695427415667065, P142=0.00040101841917697986, P143=0.0004226987478140747, P144=0.00015323782492789978, P145=0.00011316663257288412, P146=0.00013661427235747177, P147=9.566578828078043e-05, P148=0.00020733483045147198, P149=0.00023987283792835612, P150=0.00019967000151553454, P151=0.00016769720169278917, P152=0.00010484976989535745, P153=0.00010899290686082087, P154=0.0001385268064653171, P155=0.00018521717381634256, P156=0.00021790553370203207, P157=0.0001827820710409923, P158=0.00022580873403772527, P159=9.613780058653269e-05, P160=2.8904157298870584e-05, P161=2.1460503603123932e-05, P162=1.1562277371946538e-05, P163=6.541839946800024e-06, P164=7.314788283014269e-06, P165=9.943971580093562e-06, P166=1.596432193403712e-05, P167=1.4744628614391783e-05, P168=1.5171417025223467e-05, P169=1.731779365868768e-05, P170=2.0871367188714717e-05, P171=3.0859353664081253e-05, P172=3.119390167486194e-05, P173=3.065461750302843e-05, P174=2.8512891207184064e-05, P175=3.2723986527261464e-05, P176=3.4205424043161094e-05, P177=3.838037675544716e-05, P178=2.001266489932278e-05, P179=2.7405831888372616e-06),Row( P0=-671.7143574391957, P1=33.42072711215608, P2=-13.277628255859993, P3=-11.379275822157052, P4=-19.350936613129477, P5=-10.543511610600158, P6=-9.402894168090898, P7=-6.927037108642957, P8=-2.805486961214141, P9=-4.330172999028499, P10=-5.4681267414238235, P11=4.521333831585526, P12=2.0384645731817845, P13=2.792695983904262, P14=3.545984669965308, P15=-0.10432128732811015, P16=-1.7413101710990198, P17=0.9195673288664277, P18=0.9802046651717121, P19=0.5304387716934569, P20=-1.9061626769230346, P21=2.7931735796605013, P22=-0.28988068654980736, P23=1.8130148122921501, P24=-0.35400665353890226, P25=-0.5238997343177646, P26=-0.0796205941256552, P27=0.38222046844755364, P28=-0.4268431606174009, P29=0.9712357500883512, P30=-0.28415973438522607, P31=1.2467494003238682, P32=0.9340144735969592, P33=-0.45074074378682566, P34=-0.38695704555168653, P35=-0.016962115193248535, P36=-0.9255141019542961, P37=0.6225054542246767, P38=-1.249060023399829, P39=0.6416158445506589, P40=0.5486525315923806, P41=0.5497485494465508, P42=0.5661162272134591, P43=0.5422571523175891, P44=0.5508186735517198, P45=0.5364365182997607, P46=0.5570231249938002, P47=0.5287562193744909, P48=0.5301018400190416, P49=0.5246825377142732, P50=0.5051284368000866, P51=0.5284566600388625, P52=2.2711599475659678e-05, P53=1.883626517979456e-05, P54=1.2980666328112516e-05, P55=6.903448513911951e-06, P56=1.1966430316926815e-05, P57=1.897932316309626e-05, P58=1.187208636927842e-05, P59=1.60460311787908e-05, P60=4.0167852625668666e-05, P61=7.440738389779121e-05, P62=0.00018545638653174085, P63=0.0003846901909701554, P64=0.0005949963941171306, P65=0.0018546825233814901, P66=0.004558690678631827, P67=0.0061099422099588855, P68=0.0051042470712556924, P69=0.017395181094610405, P70=0.025150986678789775, P71=0.017112482596724536, P72=0.02795605438309238, P73=0.027732430534889163, P74=0.028483999557985928, P75=0.031153386846392905, P76=0.02047513140516208, P77=0.014955736814197295, P78=0.014483062954338485, P79=0.0179247207333184, P80=0.02110772366241861, P81=0.012289259896580838, P82=0.006667678371971812, P83=0.0074904629024624405, P84=0.010192300812322466, P85=0.015529860061573812, P86=0.01543748180091628, P87=0.01419817596675135, P88=0.013779461460052022, P89=0.009225444294805155, P90=0.006243207963103788, P91=0.006512178535985723, P92=0.005418550840739995, P93=0.00266509160147037, P94=0.003785267737643848, P95=0.00464936117081815, P96=0.0023248549531282286, P97=0.0021927295398583677, P98=0.0016455451970456112, P99=0.0011404371304499345, P100=0.001290272409109654, P101=0.0012603441819604576, P102=0.0004872398921723211, P103=0.00022829193951308594, P104=0.0005479848461916729, P105=0.0009799859291675466, P106=0.0012704543765531709, P107=0.0007057312304812399, P108=0.0006733418129456672, P109=0.0018985054992526227, P110=0.0017451515934579181, P111=0.0010311380039907032, P112=0.0008107479805749558, P113=0.00033879780398676643, P114=0.00032861354060807974, P115=0.0003427167518978323, P116=0.00035408600089458354, P117=0.0003498917750701199, P118=0.00019267167463051478, P119=0.00026396553326236136, P120=0.0005463094363636274, P121=0.0006580580743175879, P122=0.0005269981846754306, P123=0.0007648928833828353, P124=0.0018397374036432646, P125=0.0015962634565058424, P126=0.0007238053564551926, P127=0.0007109759434895998, P128=0.00040510346119369876, P129=0.0004394663692501811, P130=0.0006839528250623816, P131=0.0010944296661205462, P132=0.0013122657811122738, P133=0.0014137363209560832, P134=0.0008966605130563885, P135=0.0011246468306979097, P136=0.001000104512154686, P137=0.0007294977167526679, P138=0.000580508626406712, P139=0.0011803108261885195, P140=0.001394449307015802, P141=0.0016209427594395716, P142=0.0017219638548861965, P143=0.0010103848813964877, P144=0.0008542220117631669, P145=0.0010427403535376038, P146=0.0009398601928260678, P147=0.0005887176129714105, P148=0.0006978889667473329, P149=0.0007272486384338844, P150=0.0009254066642781985, P151=0.000657253643544071, P152=0.0009140107540454736, P153=0.00045861604717255695, P154=0.00034786644167012187, P155=0.000300781172454261, P156=0.00025649921152273226, P157=0.0004044759285868146, P158=0.0003098560325526131, P159=0.000426766357065713, P160=0.0003132303896355669, P161=0.00034485942689688333, P162=0.0003724976393051506, P163=0.00032069156444453534, P164=0.0003773282315628559, P165=0.00016677875928509963, P166=0.00015079898427736608, P167=0.00018864860775198185, P168=0.00018780147170551983, P169=0.00014660397082747302, P170=0.0001307601660178697, P171=9.52817241567046e-05, P172=4.863679221252894e-05, P173=6.239962313882487e-05, P174=8.553403075039443e-05, P175=0.00011226189698080234, P176=9.154444533941365e-05, P177=0.00013188616438461655, P178=0.00012094025976340133, P179=1.999002819854489e-05),Row( P0=-844.475378248368, P1=40.09552370963387, P2=-12.594985469611125, P3=-2.333894602753827, P4=-18.25412290046622, P5=-11.397814654505371, P6=-7.102619282313642, P7=-3.626620437966126, P8=-4.455754418212497, P9=-6.873876769349771, P10=-3.100374348282606, P11=-2.810867399330363, P12=-1.5341604761816539, P13=-0.2340744547512163, P14=-0.12329076909903434, P15=-0.12713274398917584, P16=-1.7636182775255393, P17=5.398718154209164, P18=3.3175590359510174, P19=9.328056453457844, P20=4.966419592115658, P21=6.492343867030669, P22=1.0226727735405061, P23=-3.178710335797803, P24=-0.5280701200730947, P25=-1.4928352680097816, P26=-0.5840169764785569, P27=0.8618862438350373, P28=-3.5399500657258165, P29=-1.1163572763441656, P30=-2.931410831694886, P31=-2.7778934947073526, P32=-1.4076159037524583, P33=-1.7625524275543756, P34=-2.159222439836055, P35=-2.8875395158524784, P36=-1.883912966057478, P37=3.7367293469005336, P38=4.982208967024079, P39=6.302943483221653, P40=0.4531912587114469, P41=0.45764529906307994, P42=0.414299113781871, P43=0.43335704674663506, P44=0.5178495290320296, P45=0.4629319645592992, P46=0.431375448932341, P47=0.4379064984593603, P48=0.5436778790346563, P49=0.6129814647787123, P50=0.5098841981263428, P51=0.4488928719536879, P52=5.301835634751219e-07, P53=6.243208629207119e-07, P54=2.781342727514741e-07, P55=7.669570893718918e-08, P56=6.007519069692015e-07, P57=7.469120054320297e-06, P58=9.41619362058421e-06, P59=2.057265890974685e-06, P60=4.770206383460113e-07, P61=2.597702029133142e-06, P62=2.767928754234437e-05, P63=0.000405715809458618, P64=0.0021072895978805437, P65=0.0027871573421879574, P66=0.0013006419370868607, P67=0.0008404557997306627, P68=0.00024825269644295725, P69=5.192917313485154e-05, P70=6.20230842362908e-05, P71=0.00047731826517941154, P72=0.0010857636819573452, P73=0.0004554863017562841, P74=8.543121651381918e-05, P75=4.235571334688687e-05, P76=9.610212164185423e-05, P77=0.0008685406404260435, P78=0.0016025537044782215, P79=0.0008830215752163877, P80=0.00022382404929919552, P81=9.419115653996718e-05, P82=0.00015242511055914222, P83=0.00018857951443565426, P84=0.00016181349817842412, P85=0.00013686235028969207, P86=4.1918208224857896e-05, P87=1.718845255447115e-05, P88=2.1984467522011273e-05, P89=0.00016498980141173042, P90=0.0005812328855537102, P91=0.0012110841129814938, P92=0.0003427182570107575, P93=3.688609255877768e-05, P94=8.951217708155845e-05, P95=5.691867069860192e-05, P96=8.924516346058398e-06, P97=8.309019876282957e-06, P98=5.316908505401174e-05, P99=0.00011603274428745399, P100=7.449932068911176e-05, P101=4.689597889616433e-05, P102=2.8297518509738557e-05, P103=3.0038995411869e-06, P104=3.7956670299691587e-06, P105=2.021927540525895e-05, P106=3.8804186639181076e-05, P107=2.3132965202657882e-05, P108=1.0682794486511384e-05, P109=1.9620327297208196e-06, P110=4.311858286861847e-06, P111=2.146018832645208e-05, P112=1.5236216465406116e-05, P113=4.312494895794428e-06, P114=3.358645278751602e-06, P115=7.129141165288768e-06, P116=1.2620149287464445e-05, P117=7.963064703758139e-06, P118=1.0028394751920535e-05, P119=4.976475834409543e-06, P120=1.473037898049277e-05, P121=1.1065215056424972e-05, P122=3.5186571937581677e-06, P123=5.192231947411045e-06, P124=8.286220999875337e-06, P125=8.370172181183195e-06, P126=4.571343198025659e-06, P127=6.559968301669388e-06, P128=4.904226953765183e-06, P129=7.306609640512262e-06, P130=7.779162350463386e-06, P131=1.6111770001236278e-06, P132=1.4147639916353297e-06, P133=1.9059895971988265e-06, P134=1.8026756424590021e-06, P135=2.1320710455741484e-06, P136=2.016159138019723e-06, P137=2.379705857798436e-06, P138=2.5271081959561635e-06, P139=1.7984714330675651e-06, P140=2.1858307503985504e-06, P141=1.5581281918433766e-06, P142=1.0816356092544572e-06, P143=1.0859909598396917e-06, P144=1.4414423802824015e-06, P145=2.876300063285556e-06, P146=5.653200920570906e-06, P147=5.614856980544791e-06, P148=4.372414553145213e-06, P149=3.3930880432812544e-06, P150=3.3878167809896016e-06, P151=1.1875055210564432e-06, P152=6.090495453542557e-07, P153=1.0674851439150126e-06, P154=1.6410325104105554e-06, P155=1.933610632741495e-06, P156=2.033358189699578e-06, P157=2.3085154512304644e-06, P158=1.679817866169012e-06, P159=1.6394308027619826e-06, P160=1.7105597364193605e-06, P161=1.5488569790288919e-06, P162=1.1869636590628022e-06, P163=6.203414235879015e-07, P164=3.829662299438442e-07, P165=4.302961761590789e-07, P166=2.5914379059561553e-07, P167=2.8458240317437037e-07, P168=3.5811149371443596e-07, P169=3.6768447962445473e-07, P170=5.219306819354593e-07, P171=6.382679318922961e-07, P172=7.694213088086937e-07, P173=5.08262420786551e-07, P174=6.604806881878181e-07, P175=6.49757670066567e-07, P176=6.332571789417724e-07, P177=5.341007363946843e-07, P178=5.594517635398236e-07, P179=1.3183712426221836e-07),Row( P0=-465.54413605007176, P1=-10.153178708238936, P2=-48.145446940150165, P3=-18.846609442779318, P4=-17.87487719391711, P5=-3.7760440434863627, P6=8.08965669838704, P7=8.530944102029633, P8=7.4760785341516565, P9=-7.763054619649895, P10=-10.093109239123791, P11=-1.5681426693078122, P12=4.489328106404251, P13=11.114624239787979, P14=-0.9194849597106552, P15=-2.7671726506445395, P16=-8.213545577404629, P17=5.042368256780223, P18=1.7266558031374537, P19=-0.6964993762265206, P20=3.721029709292085, P21=-2.761894803168869, P22=-3.256763460308177, P23=-0.42958056500955133, P24=4.921630566798392, P25=-0.778983885816776, P26=-3.043507042428571, P27=0.9140622859999784, P28=-1.6315499909416034, P29=0.5892240741092636, P30=1.207488905484939, P31=0.8306511402106287, P32=-0.34513577584340654, P33=1.007713833187346, P34=-0.384571001497253, P35=1.090287356978076, P36=-2.908631249520936, P37=1.5099531163110727, P38=0.16609545222476366, P39=2.5355833077983285, P40=0.6536823350510503, P41=0.6643544613711541, P42=0.5988473949651096, P43=0.5877491385103127, P44=0.6719462083423071, P45=0.7010676139122219, P46=0.6747086611406569, P47=0.6625963317300705, P48=0.6257732729456245, P49=0.5382018284156864, P50=0.5214159928718551, P51=0.5644210021181297, P52=1.0463306377106093e-05, P53=9.428224082741585e-06, P54=3.890826281749504e-06, P55=3.3909913657605887e-06, P56=9.14456049183347e-06, P57=1.6430026635086145e-05, P58=1.3781202899906331e-05, P59=1.228811848184708e-05, P60=1.1115071707845837e-05, P61=1.1367094563830292e-05, P62=1.7078869572345162e-05, P63=2.6337222388755518e-05, P64=2.828438359110732e-05, P65=9.521247902810319e-05, P66=0.000920930155489909, P67=0.00210636382743409, P68=0.0013963263808278428, P69=0.0011993208072450749, P70=0.002848367691939818, P71=0.015876205479689917, P72=0.03484760392058873, P73=0.030209345191743536, P74=0.019713321520220876, P75=0.024306895851015288, P76=0.021236509819175665, P77=0.0230928674374211, P78=0.01991686710601076, P79=0.016546575760830226, P80=0.014245203219061461, P81=0.011637772430633707, P82=0.01243416380355848, P83=0.02681302117191613, P84=0.07004722728606637, P85=0.0948285282709587, P86=0.13928387896235467, P87=0.17146244742337796, P88=0.2361906807294855, P89=0.47816155714283487, P90=1.0607141846634929, P91=2.0420960133178085, P92=2.217930741371434, P93=1.3820641595214247, P94=2.5684170274896916, P95=3.1673395100599966, P96=1.7916793212809026, P97=3.357481291004258, P98=0.6205784321251938, P99=0.05244651747045974, P100=0.038604704932367326, P101=0.02522748863100242, P102=0.06855848101486665, P103=0.09724839164539761, P104=1.0174777407435713, P105=8.289834788872374, P106=3.4447348046333177, P107=0.29628268083812753, P108=0.1776374619017817, P109=0.2236979806889404, P110=0.2606082920446636, P111=0.32412896505198857, P112=0.2977064041897339, P113=0.36268398683474495, P114=0.4833652656773455, P115=0.7901669253085183, P116=0.4367366992480416, P117=0.3837534872133789, P118=0.8657986907354278, P119=0.41550399240407543, P120=0.13633043651966362, P121=0.025941012662561314, P122=0.015898981272276198, P123=0.03817495367460984, P124=0.04326626567238766, P125=0.02938968334332921, P126=0.12489677472260186, P127=0.1438655835689384, P128=0.35606693013121493, P129=0.286508191261571, P130=0.01579558565641979, P131=0.006615991113116148, P132=0.017985463076110906, P133=0.01780651948689798, P134=0.008404599298914883, P135=0.009354770566429438, P136=0.019483006049834904, P137=0.052091564086410844, P138=0.030746796893203143, P139=0.03752956418699517, P140=0.026189366892925024, P141=0.017272864333140153, P142=0.01492170188883588, P143=0.01281950721633272, P144=0.020868083831528984, P145=0.005342921327221012, P146=0.004117769230942802, P147=0.006888776898067714, P148=0.008087164460054891, P149=0.0050565488595952154, P150=0.007546136537937389, P151=0.006701122468837863, P152=0.00472469255325385, P153=0.004591136343662674, P154=0.011010521222276897, P155=0.008140266584699823, P156=0.004340841811218237, P157=0.0031874339660352457, P158=0.00136016162813165, P159=0.0006327035612073676, P160=0.00047797501880170336, P161=0.0005049829079796184, P162=0.00044455284484601066, P163=0.0003147337494871639, P164=0.00024956739618339394, P165=0.00028821285304008105, P166=0.0005557627263029466, P167=0.0010643481495844094, P168=0.0017102283241321882, P169=0.0013288718101499154, P170=0.0011485344471847724, P171=0.0009923006109595542, P172=0.0009014198939885446, P173=0.0010313679347642407, P174=0.001024407137243506, P175=0.0011208791895525928, P176=0.0009719421345099108, P177=0.0011602845222452264, P178=0.0005724256575233138, P179=5.989214159630666e-05),Row( P0=-630.733844046048, P1=18.724152730958703, P2=-19.17481688815647, P3=-9.804791867652852, P4=-16.934214630865537, P5=-11.53961402773143, P6=-7.654144552233996, P7=-7.9149696535543415, P8=-6.535598427163507, P9=-7.627151056008162, P10=-5.316367183394903, P11=-0.07844615759251185, P12=3.6020589754348293, P13=7.104591493532774, P14=4.9538281849096, P15=5.755879706086198, P16=0.2979404800706753, P17=0.9983928817040685, P18=-2.7028153526053957, P19=-4.066472317340856, P20=-4.060495343869241, P21=-1.72780684793221, P22=-0.7077175574058797, P23=-1.7880284453972353, P24=-1.953391982057582, P25=-4.014893403691681, P26=-2.5058858975719662, P27=2.4047573883004847, P28=1.6576109215905235, P29=1.934796063639205, P30=-0.2187675032458164, P31=1.6056737159377623, P32=2.8796345373329664, P33=0.7216556612012144, P34=-1.0174829937716814, P35=-1.0525719129614317, P36=0.33535271384770127, P37=-0.13710040380776306, P38=-0.9612000533010036, P39=0.5157497553358088, P40=0.5696313602420495, P41=0.612267948243835, P42=0.6762653571736904, P43=0.6462762923151663, P44=0.6059473697315322, P45=0.5945386366673554, P46=0.5836517381991924, P47=0.6009859271221107, P48=0.5505415063782092, P49=0.5119239325477467, P50=0.4999463456282626, P51=0.5233159074959447, P52=9.775690492156713e-07, P53=2.1779948002165346e-06, P54=2.99895291238202e-06, P55=1.1566108504911433e-05, P56=2.3137828974345195e-05, P57=5.705715977580225e-05, P58=3.964048314485103e-05, P59=3.7258414559915945e-05, P60=4.6279888397200947e-05, P61=6.549567308584281e-05, P62=9.276207329637822e-05, P63=5.4782153514896025e-05, P64=4.9919753094175184e-05, P65=0.0004389123686964866, P66=0.003182811615036968, P67=0.008803424905708119, P68=0.028951904858988024, P69=0.07953840941698694, P70=0.06218316249438584, P71=0.03244940165457381, P72=0.02212331976900539, P73=0.016954500121887057, P74=0.01357135001372464, P75=0.014328250892757712, P76=0.009573142744465139, P77=0.008539423909778536, P78=0.009305163026804921, P79=0.004834545923266757, P80=0.0012822781083672735, P81=0.0006577308635784342, P82=0.0012289461327958675, P83=0.007285202150407361, P84=0.026766010002436027, P85=0.10429574387895824, P86=0.3265175496453125, P87=0.35803116354675285, P88=0.1527107893240244, P89=0.036004948594643954, P90=0.011491721096169529, P91=0.00856393401852744, P92=0.009037324373910288, P93=0.011682661096416375, P94=0.010055125895712268, P95=0.01837400001347772, P96=0.0346071791874599, P97=0.07006696486473507, P98=0.0768272262256727, P99=0.041170497202980466, P100=0.01951756654747665, P101=0.006089781118693322, P102=0.0005265899252702367, P103=0.0001619757836647887, P104=0.00152658815355628, P105=0.037216118462178024, P106=0.03234567526293907, P107=0.015829577400883425, P108=0.015262648843333908, P109=0.021153086495550914, P110=0.03997827635451744, P111=0.03478761147339148, P112=0.028725399658062135, P113=0.007533351566789247, P114=0.008357770426125798, P115=0.0053157496693000735, P116=0.0037602990958136512, P117=0.007592454346624018, P118=0.0023258044543785607, P119=0.0012101588980015603, P120=0.0007259921951173494, P121=0.0006058073459620513, P122=0.00045705170630269286, P123=0.00038096146805497924, P124=0.0006280879338183486, P125=0.0005665594623452257, P126=0.0009399059705472407, P127=0.0005944571795974479, P128=0.0004953741251926104, P129=0.001057421558744672, P130=0.0008374082315044761, P131=0.0003638437176159622, P132=0.00046347785926882814, P133=0.0004953620128150887, P134=0.0007594261345923738, P135=0.0008668816127055536, P136=0.0005221262490109462, P137=0.0007809394312913621, P138=0.0008043861832294999, P139=0.0006441466417577, P140=0.0006254523398754101, P141=0.0003794686690938685, P142=0.0004575849915077736, P143=0.0002873265995650833, P144=0.00019988093954938945, P145=0.00023830444212397087, P146=0.0002758072693453425, P147=0.0002865275386187056, P148=0.0003526410423538143, P149=0.00034236720155327504, P150=0.00034593835513235257, P151=0.0007346288567207126, P152=0.0002956780351301002, P153=0.0001054944648675329, P154=0.00015352462668716596, P155=0.00024545031879046313, P156=0.00043015733506037915, P157=0.0007594908258512317, P158=0.0006713232817127099, P159=0.0004243219373962647, P160=0.0003257701076616506, P161=0.00032748760260265154, P162=0.00026796451039503004, P163=0.0002677267773036578, P164=0.00033248625072614384, P165=0.0003176303507511336, P166=0.00043744230116175083, P167=0.00023352232670640347, P168=0.00019824677257883845, P169=0.00020296425670678455, P170=0.0002226506570770774, P171=0.00020866674536154694, P172=0.00015543768168538029, P173=8.063284788094229e-05, P174=0.0001119412926474281, P175=0.00012959386479591148, P176=8.290162063884232e-05, P177=7.308698586819308e-05, P178=3.86696758003528e-05, P179=5.304234571379353e-06),Row( P0=-662.2652334387569, P1=35.56273257349178, P2=-13.276827566070148, P3=-4.473740370689278, P4=-22.578452617673975, P5=-15.362077308631864, P6=-12.751972587922115, P7=-7.982913920072315, P8=-5.73916517543822, P9=-12.44765056646546, P10=-8.72082366003256, P11=-8.777115215465804, P12=-7.72673337708485, P13=-2.128147048501512, P14=-3.1189084244749656, P15=3.9651658303956574, P16=0.6455319133529095, P17=10.229673315198424, P18=4.939402482239117, P19=-0.6381748113474198, P20=2.9653523388898706, P21=1.2818948047493015, P22=-0.49050310286630294, P23=4.071500874694754, P24=0.8901351849682745, P25=-2.5657559099347313, P26=-4.776946884469176, P27=-2.7164190183989128, P28=-0.9844204104336196, P29=-0.04914572066063461, P30=0.7171900260594593, P31=-0.9429248329070994, P32=-0.8099510671836295, P33=1.9316559476750979, P34=-0.5790074158612804, P35=-1.1206922428297181, P36=-5.971959200555708, P37=-0.28464052750556806, P38=0.6097704925021706, P39=0.5160303635950978, P40=0.7290202221705028, P41=0.7252088951028027, P42=0.6185409608825078, P43=0.5651344563143486, P44=0.6038915400063365, P45=0.5913090900037761, P46=0.5675527832527492, P47=0.6203023297113384, P48=0.6105267809230187, P49=0.5713030438108827, P50=0.5852471214227594, P51=0.616598878773825, P52=4.0520459511378066e-07, P53=1.2917224878801435e-06, P54=1.1715476387867474e-06, P55=2.4949961565867447e-06, P56=1.0562946334587737e-05, P57=1.3713316573903599e-05, P58=5.800057424258468e-06, P59=3.75338673092523e-06, P60=9.088612256154065e-05, P61=0.0010674651851814425, P62=0.004108801563433615, P63=0.009481527620784908, P64=0.008491289522868744, P65=0.03784765894133439, P66=0.07080919442884666, P67=0.08197791671405485, P68=0.04913243823048764, P69=0.025638834148718877, P70=0.009312349132124259, P71=0.0014162481041014115, P72=0.0007216702649877134, P73=0.0008415344036841995, P74=0.004388831140515208, P75=0.00915590112163855, P76=0.010700164488042604, P77=0.01547294030538333, P78=0.012598585960257766, P79=0.00834758618351397, P80=0.014094040884249894, P81=0.010589328744882052, P82=0.007259940125616803, P83=0.015739106666537193, P84=0.04770977073432778, P85=0.0626499672624725, P86=0.033286448945653836, P87=0.02148717810096483, P88=0.013114463455999184, P89=0.00797508152983491, P90=0.00826628280361144, P91=0.010323121861656325, P92=0.005929798566227472, P93=0.00224996805093607, P94=0.0010519837591462403, P95=0.0028963063334528695, P96=0.05846394697815606, P97=0.06351703718573584, P98=0.01633470938824344, P99=0.002889404542798345, P100=0.0021053286058569463, P101=0.0005262623843345404, P102=9.88369686447436e-05, P103=4.878464323954386e-05, P104=0.003403986614709618, P105=0.0811330516482203, P106=0.02351368429122027, P107=0.0005387522712174554, P108=0.00025080081933752396, P109=0.00039051368785995394, P110=0.00017887081869204824, P111=0.006330554536147738, P112=0.05739674234070622, P113=0.008233245030608485, P114=0.0019522094164346041, P115=0.0003726532580965011, P116=0.0010651526719083837, P117=0.01582753854138367, P118=0.008157348917740488, P119=0.0007873177317795938, P120=0.0002980680769939866, P121=0.00046995072588656143, P122=0.0007173374065593458, P123=0.001366155178351806, P124=0.0009536194468312103, P125=0.0002904261826123149, P126=0.00034283140894623645, P127=0.0006320652258620848, P128=0.00029356458763478264, P129=0.0001406311369284885, P130=0.00018287523821782726, P131=0.00010872255489528745, P132=5.550803378047289e-05, P133=0.0001400591751098382, P134=0.0001779051888712678, P135=0.00018549319540247535, P136=0.0002102691376972653, P137=0.00030268899114553886, P138=0.0004607037740249124, P139=0.0009237545065389724, P140=0.0004711163628271498, P141=0.0014604707305296458, P142=0.0005004185104219138, P143=9.255423229683087e-05, P144=9.501202242923552e-05, P145=6.0682486112889226e-05, P146=9.832490842366411e-05, P147=0.00014652603312347416, P148=0.0003158597517057408, P149=0.000407942874102387, P150=0.0003858955895588335, P151=0.0003007558177714454, P152=0.0002932811260077129, P153=0.0005933853469854843, P154=0.0006662954068195816, P155=0.0005774229152549372, P156=0.0004169907038419421, P157=0.00013662428160070595, P158=0.00012193604783801252, P159=0.0001283734195865871, P160=0.00013900044738704702, P161=0.00010084297821327292, P162=8.808442532531179e-05, P163=7.089819645136921e-05, P164=4.185894222668425e-05, P165=3.6118838656167165e-05, P166=5.791250725156945e-05, P167=9.009119782571321e-05, P168=6.515921846838988e-05, P169=6.382004770881088e-05, P170=7.918131242818948e-05, P171=0.00011002201650228412, P172=0.00010327013688229526, P173=9.069885148561179e-05, P174=5.9171378671045055e-05, P175=5.7054662796186946e-05, P176=7.230910360637534e-05, P177=6.007616547192056e-05, P178=4.2242963093855134e-05, P179=4.328626742356755e-06),Row( P0=-773.0631674127569, P1=21.183078364354962, P2=-9.078624252984909, P3=-5.662596793416827, P4=-18.10273446162972, P5=-12.444543756005249, P6=-8.116310375126691, P7=0.3857082646860168, P8=-4.5044497871362355, P9=-11.733316130923786, P10=-1.4233610760500166, P11=-3.679290464395019, P12=-3.7993507376114573, P13=0.8962830320671683, P14=4.694152712957404, P15=6.618852496141321, P16=3.9556159544720684, P17=6.107714284230173, P18=3.6878504511554238, P19=-0.044635173841325745, P20=-1.0381201969193328, P21=-0.19816940819378526, P22=-2.1491033803017268, P23=-2.7891799166054345, P24=-3.1178078788921018, P25=-4.67392297810957, P26=-3.6303345463886236, P27=-0.8606457837188469, P28=-0.40923754701219733, P29=0.11772112411973878, P30=-1.2609764635635619, P31=0.1285251685930282, P32=0.6462517544845343, P33=-0.5145298896536294, P34=-3.6953430241894707, P35=-1.516852569564816, P36=-2.250194541979858, P37=1.8392796031156804, P38=-1.3653327750035253, P39=-0.7312785953695955, P40=0.5615260569285206, P41=0.6018376725225563, P42=0.567194761786322, P43=0.4822550768816357, P44=0.46700781089363225, P45=0.45747168683011774, P46=0.4404880174028181, P47=0.438748679726563, P48=0.4754712866948641, P49=0.47271775504471975, P50=0.4876987691751942, P51=0.5253390577445058, P52=1.2565864150211406e-07, P53=2.231105372556841e-07, P54=3.46516977544057e-07, P55=2.9239799669186375e-07, P56=8.333243391924821e-07, P57=1.7846985779588829e-06, P58=1.6702956123361076e-06, P59=1.0337757682765454e-06, P60=6.066979276130493e-07, P61=1.325954196013159e-06, P62=2.040557935393001e-05, P63=0.00019628166878940607, P64=0.0005587481575772537, P65=0.003746580073099264, P66=0.011053662518443363, P67=0.015444291317254943, P68=0.023956291722471268, P69=0.014052398235771955, P70=0.0038817467364094414, P71=0.0005614592896658686, P72=8.984255113889e-05, P73=5.2560316045891875e-05, P74=4.806595098820162e-05, P75=9.66510472014901e-05, P76=0.00030277232461637006, P77=0.0005829709353711409, P78=0.0007888761163671144, P79=0.0012127298236399413, P80=0.002914756925450312, P81=0.00490692174938751, P82=0.003687702169390222, P83=0.001686021899999127, P84=0.0055977931169396205, P85=0.013725698649878408, P86=0.011447655811660976, P87=0.005809414624257841, P88=0.002866043435833403, P89=0.0010996213309077609, P90=0.00076463584473292, P91=0.0006113111812974652, P92=0.0006897936916573309, P93=0.0008567054763804167, P94=0.0015121295324680875, P95=0.0006017545701582694, P96=0.00010087630238840136, P97=0.00032145519920020717, P98=0.0005128112234975025, P99=0.00030368956399260674, P100=0.00020241188717859607, P101=6.503740615941939e-05, P102=0.0001148365400628978, P103=9.591013885847036e-05, P104=3.058318817861109e-05, P105=2.0999976434857042e-05, P106=2.7323013968292312e-05, P107=2.644866045994151e-05, P108=2.211151896663181e-05, P109=2.9555160274894164e-05, P110=4.9150474709648105e-05, P111=2.256466318259708e-05, P112=1.247040275945595e-05, P113=9.300393369975856e-06, P114=3.248064471288557e-05, P115=5.4549792291723774e-05, P116=3.4512718021792696e-05, P117=2.183383186740885e-05, P118=1.0142213169492204e-05, P119=2.5148954990785054e-05, P120=0.00013184386278504396, P121=5.523629318722846e-05, P122=1.2080396189973426e-05, P123=5.489319411925755e-05, P124=0.00012199527783683028, P125=9.779624043478403e-05, P126=5.818650240960254e-05, P127=7.836222922466465e-05, P128=0.00013556876506131698, P129=4.0813832248773326e-05, P130=1.122705907806095e-05, P131=1.347868118389453e-05, P132=1.783522746283216e-05, P133=6.708859470046747e-06, P134=8.113257425360906e-06, P135=1.1041028818227762e-05, P136=1.0147790236939578e-05, P137=1.340425238267977e-05, P138=1.1873315555810602e-05, P139=9.644774381444766e-06, P140=1.0240988652645462e-05, P141=6.878036829185972e-06, P142=8.067384148314285e-06, P143=4.580492337783972e-06, P144=4.913580640788146e-06, P145=6.433984340661899e-06, P146=6.732015420268887e-06, P147=8.009677511816852e-06, P148=8.509185303654561e-06, P149=1.6607295743880413e-05, P150=1.7935364050769077e-05, P151=1.714618912915098e-05, P152=1.6724527721630894e-05, P153=2.4603077123715913e-05, P154=3.400065541544485e-05, P155=5.964916246578965e-05, P156=5.4030939242665424e-05, P157=7.380016738823797e-05, P158=4.7768175597894794e-05, P159=4.20633038275265e-05, P160=2.9745054004911312e-05, P161=2.6396918453643064e-05, P162=1.4675033728475866e-05, P163=1.0124326007983841e-05, P164=7.475973249740905e-06, P165=1.4109567485752947e-05, P166=1.2010393028695154e-05, P167=7.344399296319015e-06, P168=5.369806764651247e-06, P169=7.1766174068158315e-06, P170=7.3997076998729375e-06, P171=6.362042915299088e-06, P172=8.462365877894769e-06, P173=7.112543544813634e-06, P174=5.884233366654049e-06, P175=6.344136025768917e-06, P176=7.415644580716307e-06, P177=9.834970880811186e-06, P178=6.242434284249721e-06, P179=9.110552254405855e-07),Row( P0=-786.766982858639, P1=33.38142100966118, P2=-13.356117444498134, P3=-7.176221980284061, P4=-24.62671035054522, P5=-14.795269700928802, P6=-8.96085993819566, P7=-0.28572799486329575, P8=-3.789272251117247, P9=-13.140749215350866, P10=-3.468682883992025, P11=-2.9505979075885143, P12=-3.8858796910094275, P13=4.12323013530586, P14=4.387408671846992, P15=9.81958675057383, P16=9.183480033612458, P17=13.719085000564492, P18=12.83694388091762, P19=8.198795206281025, P20=2.9679457339387194, P21=2.904962706436448, P22=-3.1527456716673274, P23=-2.3631687577875335, P24=-1.3415259012017697, P25=-2.918339180962781, P26=-2.6428673767581614, P27=-3.620210080170867, P28=-5.820319137878768, P29=-1.4088839294011795, P30=-2.6915480396579654, P31=-0.3558611720060825, P32=-0.9025568508868232, P33=0.5012004666983586, P34=-2.265117465080009, P35=2.45340667166359, P36=-2.98679956782431, P37=1.6698006207744598, P38=-2.1839732297455368, P39=0.9235214109654549, P40=0.5859318899889588, P41=0.5381781071605498, P42=0.47199883018948086, P43=0.4532484402312305, P44=0.46121855703348646, P45=0.42460349515322054, P46=0.3930867850043264, P47=0.39127297579644554, P48=0.4130600870719989, P49=0.46437873566888627, P50=0.5142690849119262, P51=0.5663757682493951, P52=3.1099564634266744e-07, P53=1.0024746474535317e-06, P54=5.235741910889578e-07, P55=4.0660032155730866e-07, P56=4.930802964318219e-07, P57=2.77822474540241e-07, P58=5.4395065138654346e-08, P59=2.649116992704424e-08, P60=5.47570699116479e-08, P61=2.8075231519598035e-07, P62=5.28099129016583e-06, P63=5.8799189143891065e-05, P64=0.0008272096061735462, P65=0.005725460130085586, P66=0.01180628534094577, P67=0.009587043593186434, P68=0.003597339917650931, P69=0.0012109468884868525, P70=0.0005245435285420262, P71=0.00012609699912588413, P72=6.0684865107540445e-05, P73=4.065611859920598e-05, P74=1.633542227173694e-05, P75=1.1632227649281617e-05, P76=2.390508529666881e-05, P77=0.0004526391313426956, P78=0.001538642296921111, P79=0.00126112178176123, P80=0.0014212540946796546, P81=0.0012871472159703436, P82=0.0033365368774664005, P83=0.003974770700017755, P84=0.002635086705557163, P85=0.003018095239343275, P86=0.0010547018850879906, P87=0.00028088521012912035, P88=0.00010818248748482387, P89=0.0003706811224698343, P90=0.0016649640525392287, P91=0.001396731084814262, P92=0.0003920027558954984, P93=0.0001470807315984224, P94=0.000374878191691382, P95=0.001396178726737638, P96=0.0004463637392338263, P97=0.00023786693657996906, P98=0.000280417082658292, P99=0.0002404229987507321, P100=4.903754361549654e-05, P101=1.7705293186734958e-05, P102=2.493789362167278e-05, P103=2.6181994469707716e-05, P104=5.6329806612914484e-05, P105=3.234271169511178e-05, P106=4.212170487554876e-05, P107=8.510249196996202e-06, P108=3.3541981358947397e-06, P109=5.853943378920186e-06, P110=4.7585857734349454e-05, P111=9.442904067975468e-05, P112=3.871611835103942e-05, P113=5.315513388173402e-06, P114=4.7708554245179755e-06, P115=7.3571379646377145e-06, P116=1.1004416905154256e-05, P117=7.505801241916675e-06, P118=3.877383619087616e-06, P119=7.071199381542141e-06, P120=3.877357589500513e-05, P121=2.0360651071090973e-05, P122=1.6787328290662545e-05, P123=2.2950133782016873e-05, P124=3.576438974586488e-05, P125=2.017969999273203e-05, P126=2.0152604535956474e-05, P127=1.8288360300081455e-05, P128=1.215907495136837e-05, P129=6.657941577288543e-06, P130=1.2080095698979316e-05, P131=9.06995235759721e-06, P132=4.601325349148838e-06, P133=6.253234112736804e-06, P134=1.6490427809452894e-06, P135=3.0969347853177642e-06, P136=1.952611569995221e-06, P137=3.3802615808058083e-06, P138=6.54091104503985e-06, P139=6.224555407048153e-06, P140=6.898690625045701e-06, P141=2.629815737846758e-06, P142=2.3252966964996667e-06, P143=2.7732623462833103e-06, P144=2.249770576781325e-06, P145=3.2933585521364025e-06, P146=5.89507072201587e-06, P147=1.1520668995734488e-05, P148=1.6444979031971775e-05, P149=2.1447791199958994e-05, P150=2.2436743164741924e-05, P151=2.132026383951146e-05, P152=2.786496309069876e-05, P153=2.3689399684306535e-05, P154=1.8797232615045386e-05, P155=1.2872394505950985e-05, P156=1.3070033073595303e-05, P157=1.4128960976483961e-05, P158=1.0580914270770789e-05, P159=1.019511641018623e-05, P160=1.2909447781356073e-05, P161=7.742454694516196e-06, P162=7.044759194210302e-06, P163=5.529118475339344e-06, P164=3.033212007302196e-06, P165=3.0937372807376385e-06, P166=4.261672322383317e-06, P167=5.438078396376386e-06, P168=7.4847386505720555e-06, P169=1.4320400650701528e-05, P170=1.0965705572127406e-05, P171=8.777741140061202e-06, P172=9.280851205585175e-06, P173=1.4696566992036307e-05, P174=2.4551988643184625e-05, P175=1.3773029038731714e-05, P176=1.0643041710509246e-05, P177=1.292857342725486e-05, P178=6.542060396387978e-06, P179=6.107784881879541e-07),Row( P0=-396.43637222323696, P1=7.815157278230152, P2=-50.78910143506658, P3=-10.224206420596955, P4=-20.42490850861495, P5=-6.1721658354620565, P6=1.1838239719764265, P7=4.730285973105249, P8=12.64694171634994, P9=2.7122372160593473, P10=9.321836355700462, P11=7.7743729494187175, P12=-0.768917052413213, P13=4.597641039622218, P14=-0.4546039062765214, P15=1.8234430182427102, P16=-8.069941597910502, P17=8.345020331240605, P18=3.870049376441026, P19=3.3518408433999096, P20=6.289723223831348, P21=1.4450441231284172, P22=3.435112945302086, P23=-0.21180730671348832, P24=2.4531388005966437, P25=0.5888294271752197, P26=-0.36438182959938903, P27=0.4783460661129893, P28=-3.2249624055123727, P29=2.760591773178893, P30=-3.152177565255268, P31=-0.7082540762166679, P32=2.313006653690759, P33=-0.28355601009201975, P34=-1.7142383936031431, P35=2.2549412701668086, P36=-0.9207343310394697, P37=2.6832364155399158, P38=-2.4903753688004717, P39=-0.282469420273669, P40=0.6581855983317508, P41=0.6652126686496569, P42=0.618455089268943, P43=0.5940715561803911, P44=0.5837918657456032, P45=0.5741047635119595, P46=0.5807361301850734, P47=0.5424244356950763, P48=0.5736025036657659, P49=0.6547063760405818, P50=0.652124443221334, P51=0.6410369984742205, P52=0.06285393375308237, P53=0.13641070280840473, P54=0.10008572118797861, P55=0.014941086875567763, P56=0.004413471272560418, P57=0.0009728645193954991, P58=0.0004527583359066496, P59=0.00022843529911335822, P60=0.000225830761683528, P61=0.0003246139620729759, P62=0.0005140602831946161, P63=0.0009620791124375714, P64=0.0016922515036133419, P65=0.004601794007941804, P66=0.005938304786350231, P67=0.0021278932288946316, P68=0.0013073857059524162, P69=0.0010424796271124275, P70=0.0007323311564020257, P71=0.0012250077485394127, P72=0.004630642452195158, P73=0.009847518488782491, P74=0.03684302672161355, P75=0.058469161517002884, P76=0.07510844626017232, P77=0.16569452066093163, P78=0.33159330483675753, P79=0.558571174908261, P80=0.6976614759431226, P81=1.251061835279715, P82=1.2687633440954997, P83=0.6531811966474992, P84=0.41164269603391224, P85=0.296949081950821, P86=0.29093866005826957, P87=0.38791063020540284, P88=0.5867026283669684, P89=0.8368939626811888, P90=1.4244188195369443, P91=2.4319297989690427, P92=4.260116633054771, P93=3.0050685243816724, P94=1.7075680262661246, P95=1.807278797229688, P96=2.0746545446736375, P97=4.864782749575758, P98=4.231152308599679, P99=5.096670196291638, P100=5.806946896311398, P101=4.071811144494481, P102=7.915666188094277, P103=7.146675769956963, P104=1.3783795298213992, P105=0.7217014655235293, P106=4.549053303924012, P107=4.547123943354264, P108=2.6050355348897516, P109=4.350727582729676, P110=2.5358687161393427, P111=0.5792888364988501, P112=0.9883359284855058, P113=1.2294820595047498, P114=1.2745829618853466, P115=2.147291785842714, P116=2.045073497331418, P117=1.4372681667347502, P118=0.32952613164129646, P119=0.8525270523517751, P120=0.2958471050483909, P121=0.08360419562340261, P122=0.18278904742140042, P123=0.9622690159235864, P124=0.525696542485643, P125=0.13196641471479328, P126=0.311462150787205, P127=0.25852261318470465, P128=0.054620031972440544, P129=0.06619078798540141, P130=0.06654588375348029, P131=0.03402602469987527, P132=0.03902221352979588, P133=0.050479068143814906, P134=0.03215148903536305, P135=0.0569086418627683, P136=0.07153146030031882, P137=0.05331376200963883, P138=0.0567225457361161, P139=0.052195074674877315, P140=0.07612074587092219, P141=0.047411367351210786, P142=0.027697082910932813, P143=0.019882066641357724, P144=0.012621069952660045, P145=0.014876212439681506, P146=0.011276448879671398, P147=0.01699426019214336, P148=0.026952581515810148, P149=0.015064439905960535, P150=0.01203909740009411, P151=0.011644630965707893, P152=0.010547995424306013, P153=0.013792915215113766, P154=0.016606561151271428, P155=0.00887328712409695, P156=0.0065028322467453326, P157=0.005009305963220255, P158=0.0022962850631992114, P159=0.002306452042165558, P160=0.0018162349692762434, P161=0.000918048910155323, P162=0.0007746696499923261, P163=0.0005311231337063902, P164=0.0005160754639603854, P165=0.0005595433832693214, P166=0.001235442641060208, P167=0.0015420650918216603, P168=0.001630453257329853, P169=0.0015618259628364314, P170=0.0013869445231319878, P171=0.0012576424981519958, P172=0.0010311384816789732, P173=0.0010828149586124191, P174=0.0017230754699632006, P175=0.0022954162774191968, P176=0.0026635160264208697, P177=0.0028037263587940234, P178=0.0015946961390235534, P179=0.00019350142836781608),Row( P0=-825.3645797119806, P1=27.747805383470553, P2=-16.651200160919814, P3=-4.803631351674363, P4=-19.220766150836422, P5=-12.378176353123958, P6=-10.743316668474023, P7=-6.776615832756637, P8=-7.509528306346546, P9=-10.577908996230708, P10=-7.556379884996272, P11=-3.790902128839871, P12=-3.8240025016253854, P13=-3.009134868992448, P14=3.544892304458311, P15=2.6345471279182675, P16=0.31041664845535766, P17=6.975041051927118, P18=6.087976865139756, P19=8.070817439201571, P20=7.543885386662824, P21=7.592730305661435, P22=0.6971128680282135, P23=-2.9431356210070194, P24=-3.7441862670225103, P25=-3.981356268339164, P26=-1.5542559768030206, P27=1.5291562262265774, P28=-1.3102181912164261, P29=-0.09698674454935854, P30=-2.4151644494440205, P31=-1.2808449056028566, P32=-0.0259029026808318, P33=-1.4684922669070726, P34=-0.9485136212520495, P35=-2.438835624278064, P36=-2.296701364825398, P37=0.8130602543342565, P38=0.656295003945679, P39=3.1203732768292936, P40=0.4568544092511391, P41=0.4594390478798671, P42=0.4475011674845318, P43=0.4655274585442939, P44=0.4315294015508603, P45=0.39230029812534767, P46=0.3932164177466508, P47=0.4468398972544719, P48=0.5613287905677623, P49=0.5478285738507451, P50=0.48395284053640697, P51=0.4461899743165109, P52=1.6261472149706903e-07, P53=1.2019824292573388e-07, P54=1.235734402909571e-07, P55=5.6238873041511597e-08, P56=1.6693860426870927e-07, P57=1.9108329235507472e-07, P58=5.2989114007266016e-08, P59=7.44513086095521e-08, P60=4.055948440472656e-07, P61=1.5575349388736328e-06, P62=3.835506790311055e-05, P63=0.0006422883793153003, P64=0.0024517794364480993, P65=0.0030130917799491637, P66=0.0025539561365586203, P67=0.0018218900687442061, P68=0.0012413323435328063, P69=0.0021936475749641474, P70=0.001898777278649039, P71=0.0006708880164909344, P72=9.0801299084881e-05, P73=3.076093028040805e-05, P74=1.818032928964657e-05, P75=2.9897791725140545e-05, P76=0.00020365491958496885, P77=0.0013450890293338971, P78=0.001834613020453683, P79=0.000818561934351013, P80=0.0005077863615764838, P81=0.00033680339964881767, P82=0.00021704510837596995, P83=0.00010848832918777068, P84=0.0002108079156654749, P85=0.0006734854615041262, P86=0.0010246851466917423, P87=0.0008151244857569974, P88=0.0006368890847873919, P89=0.0010995118959101153, P90=0.0015071409059632863, P91=0.0005296036064851746, P92=0.0005005870596702928, P93=0.0002848711554814568, P94=0.00021345625306799725, P95=0.00016787026710848082, P96=2.817710239685133e-05, P97=4.951785808005035e-05, P98=9.728087814063598e-05, P99=8.399317621960317e-05, P100=0.00017607053049175438, P101=0.0005833707634677672, P102=0.0003364597262863346, P103=3.436939969235759e-05, P104=2.0534586847103834e-05, P105=5.054925754403571e-05, P106=6.64543597593263e-05, P107=5.3109382292039526e-05, P108=9.404428781611118e-05, P109=4.288134827782602e-05, P110=4.28841540482092e-05, P111=9.508775241265016e-05, P112=4.274221084104571e-05, P113=2.8273744020554495e-05, P114=4.892321191770714e-05, P115=6.41841155388882e-05, P116=3.019465410592443e-05, P117=2.6005651416523132e-05, P118=4.95698966380183e-05, P119=2.845604996855242e-05, P120=2.708854187602511e-05, P121=1.872455078156962e-05, P122=2.0500185362792207e-05, P123=2.708016128568621e-05, P124=2.309975626048628e-05, P125=2.5404587528587612e-05, P126=1.785524162006207e-05, P127=1.3230327319341802e-05, P128=1.481873001335159e-05, P129=1.310545378365512e-05, P130=2.808290551435908e-06, P131=2.2623272460889214e-06, P132=2.20062891279663e-06, P133=1.4006816611562876e-06, P134=1.378797986966019e-06, P135=3.0140138378523464e-06, P136=3.1572967830141528e-06, P137=4.2912327490792465e-06, P138=5.0079977295310616e-06, P139=8.302715040947193e-06, P140=9.546328530225881e-06, P141=8.191269932238982e-06, P142=8.316372589602442e-06, P143=6.3387654134286995e-06, P144=4.796308342528706e-06, P145=3.521333893355595e-06, P146=3.784049880564092e-06, P147=3.3986537961349335e-06, P148=4.094344631425018e-06, P149=5.488555224764467e-06, P150=8.406388157305616e-06, P151=2.86314034673335e-06, P152=1.2681978348691051e-06, P153=2.5200254931574245e-06, P154=3.3422174918565684e-06, P155=3.7854683434186363e-06, P156=3.4164304926599137e-06, P157=3.708997673047883e-06, P158=2.9505189192804523e-06, P159=1.9802506582469845e-06, P160=1.6041939838448026e-06, P161=1.5162206589069995e-06, P162=1.8436434781832736e-06, P163=1.7850753899548046e-06, P164=1.2738044154050986e-06, P165=1.5544999663635537e-06, P166=2.092415174731822e-06, P167=9.042270301711523e-07, P168=8.091538692693749e-07, P169=1.161811309353339e-06, P170=1.1012424993630674e-06, P171=1.1511299902366382e-06, P172=1.2836059796895065e-06, P173=1.7834853553013822e-06, P174=1.1389002049335558e-06, P175=1.1478925595602395e-06, P176=8.829933249274168e-07, P177=1.4977034002377674e-06, P178=7.282948171725279e-07, P179=8.660196901655262e-08),Row( P0=-388.0536982335001, P1=-6.625897163119596, P2=-39.24124281421221, P3=-16.07117082414992, P4=-13.58742492866879, P5=-1.040152248383255, P6=7.880070723070356, P7=11.064086524637135, P8=10.416092917010335, P9=-3.9363122562657846, P10=-5.497962786251351, P11=3.5052201846647546, P12=1.441127295081201, P13=7.964240463832265, P14=-3.992214791753942, P15=-2.69242778448459, P16=-6.103330824761268, P17=8.404195491307172, P18=4.740064598452237, P19=-1.8714965294774035, P20=2.479470080752871, P21=1.2813585415522462, P22=-2.8961071972814683, P23=1.0827490525034729, P24=3.284666534939489, P25=-0.33717298194312284, P26=-5.6070072133801565, P27=0.893185677746049, P28=0.9368647042304659, P29=1.7728144502389382, P30=0.4392925720504641, P31=0.9726375742723754, P32=-0.522347203549477, P33=-0.6656737092158712, P34=-0.5593495453700137, P35=2.1161063389216834, P36=-1.8309202351360774, P37=0.4594708424469772, P38=-1.0643363357188915, P39=0.03756089511224897, P40=0.6053749037129744, P41=0.5700987177956325, P42=0.5200363066653931, P43=0.5284751606310768, P44=0.5399858318939655, P45=0.544742973126968, P46=0.5650066623639761, P47=0.5576986427730807, P48=0.5451538024046847, P49=0.5425150411814751, P50=0.5476485501164614, P51=0.5557525564566883, P52=0.004599012620543261, P53=0.005417316917627513, P54=0.003375752026530686, P55=0.0014451540638558708, P56=0.001030998050348823, P57=0.00046223667700014717, P58=0.0001877834908333171, P59=0.00021955430654392634, P60=0.0004018246033203163, P61=0.0003593716102633722, P62=0.0003770196227540362, P63=0.00035509593667737465, P64=0.00017882717213617925, P65=0.00085828037802631, P66=0.0017046890528991583, P67=0.003095665374742486, P68=0.0022422599413565985, P69=0.0013511748826782086, P70=0.002121092883586387, P71=0.0018015179091312014, P72=0.0006410087751219795, P73=0.0006276338743099841, P74=0.002194509478154829, P75=0.008447744845689277, P76=0.04396152120319807, P77=0.08733683140525823, P78=0.13933044719102902, P79=0.26172847618675044, P80=0.3129941898220929, P81=0.157929145935045, P82=0.07011714081912035, P83=0.0908407912532034, P84=0.2792413534055123, P85=0.4201642208038122, P86=0.24660316963397247, P87=0.3142101787838885, P88=0.5465840767155596, P89=0.7242247095393192, P90=1.575120305042242, P91=3.7506527721724923, P92=4.633603431024934, P93=8.418427495647906, P94=17.863394660272732, P95=25.369599716701373, P96=62.682120404432446, P97=45.97981667806858, P98=7.430448129295065, P99=4.862805700827098, P100=6.350455375805793, P101=3.0275867809164887, P102=1.390668430884017, P103=0.7459313444297619, P104=0.5913119084047522, P105=0.9277323111297435, P106=1.2704389995391892, P107=0.9626707213520725, P108=1.0017433131325149, P109=3.1448145189361654, P110=3.0929856647791345, P111=1.8713498985346388, P112=1.6250186806969134, P113=1.998566848049712, P114=2.1027332338692006, P115=1.986522611317228, P116=0.6219280290238959, P117=0.49488433107537994, P118=1.1356533441052175, P119=1.0411617481482374, P120=0.07615946601553028, P121=0.018693803551431652, P122=0.019516886509815336, P123=0.019689563535790152, P124=0.08723717083500111, P125=0.13168573840428627, P126=0.1694293933446878, P127=0.23521638523758123, P128=0.10060678318533649, P129=0.026454288744511698, P130=0.11300375283327788, P131=0.0735483090502825, P132=0.020148849099651454, P133=0.006502829525058736, P134=0.009983105137289918, P135=0.02703419711071667, P136=0.049541422471130116, P137=0.03749175121869324, P138=0.07798832342020179, P139=0.14891022186364908, P140=0.05370397213303933, P141=0.02781727564046213, P142=0.022671136673804656, P143=0.02410859239449256, P144=0.014038616024846703, P145=0.03051693401686899, P146=0.04889281757030555, P147=0.023203971492430784, P148=0.0264428374524912, P149=0.021781967227879617, P150=0.02217296664752827, P151=0.096630918686245, P152=0.04625686401839032, P153=0.017749786708583837, P154=0.020353773858936445, P155=0.010660543265472654, P156=0.005002934939748521, P157=0.002693393435895744, P158=0.002429155229310101, P159=0.00186406867564786, P160=0.0021812541161747795, P161=0.0016492947347444727, P162=0.0014579084017806731, P163=0.0015452410277069215, P164=0.002208108894266265, P165=0.0020588536924500754, P166=0.002946438261580193, P167=0.005707646555175158, P168=0.003882373379371062, P169=0.0039168404657650095, P170=0.003921737961614323, P171=0.0033771779012175354, P172=0.00415772386415185, P173=0.0047736504407606154, P174=0.0047621749973845, P175=0.004413294264355513, P176=0.004459420362954387, P177=0.004387112248578872, P178=0.0023751831240717293, P179=0.00028784934483936254),Row( P0=-734.6550573054161, P1=21.859891198809706, P2=-16.153965470202635, P3=-8.280056550544735, P4=-19.831440588723552, P5=-11.289301335251682, P6=-5.860378430740389, P7=-5.0137532913970295, P8=-4.155161775807018, P9=-8.128569310868482, P10=-2.4038656811879497, P11=-2.3297326075760187, P12=-1.6759907247386563, P13=4.41889775676582, P14=3.1823012325084403, P15=7.800764457404672, P16=3.8911188326157222, P17=2.639999258118831, P18=0.8520176128781991, P19=-3.1266731396010097, P20=-2.1154744056639765, P21=-1.3313902138265516, P22=-2.5512260822565866, P23=-1.1274409978398872, P24=-1.4650066674606939, P25=-3.1123637982696843, P26=-1.6917118314219675, P27=1.1841622765199018, P28=1.2370604259848184, P29=2.0267152590760613, P30=1.8506605767730273, P31=0.7379199881736058, P32=1.7107856071109606, P33=0.3769942306551964, P34=-1.1151792160856684, P35=0.24589856690381118, P36=-1.6485626032043472, P37=1.2154076438262367, P38=-1.4646681665426207, P39=0.8486032039646647, P40=0.5489935096344616, P41=0.6023233661880923, P42=0.6243025202726825, P43=0.5382986993556657, P44=0.5079181001007215, P45=0.4557988638673328, P46=0.46173464236980866, P47=0.46284384062785666, P48=0.4438362384897952, P49=0.45274057073142054, P50=0.45865168040468096, P51=0.48119218153787846, P52=2.151866736617896e-07, P53=4.376997733066536e-07, P54=1.691336052133821e-07, P55=5.291325127916269e-08, P56=4.507614696797753e-07, P57=1.4704811497502485e-06, P58=1.1196239684404562e-06, P59=1.027156546355314e-06, P60=1.589516099418779e-06, P61=1.0599750626626629e-06, P62=2.344436912377355e-06, P63=5.1310491194673924e-06, P64=5.38339833685e-05, P65=0.000468845007798353, P66=0.0029239038584475204, P67=0.007770582626708737, P68=0.025860502835545834, P69=0.08306429912721024, P70=0.04795135780662355, P71=0.0037351788526984353, P72=0.00017990426913371308, P73=0.00010012115471583612, P74=6.538963611875737e-05, P75=6.107085633272136e-05, P76=3.594071638488312e-05, P77=7.358077940284075e-05, P78=0.0002521042092574677, P79=0.00046464970007294507, P80=0.0004366656615803491, P81=0.0008469479546674104, P82=0.0023587895692916885, P83=0.006426717149092735, P84=0.009724019869958142, P85=0.007170599259865614, P86=0.0053790014502884545, P87=0.007629709837665935, P88=0.0028753646622923954, P89=0.0005370040469662139, P90=0.0008410427270365927, P91=0.0026655902071669986, P92=0.0011468678844230183, P93=0.0003135200691824537, P94=0.00025964273745725773, P95=0.0008267010175697455, P96=0.003595553516020089, P97=0.0026740939345136364, P98=0.0005379562688489255, P99=0.00016710646311103683, P100=0.00037374793516278457, P101=0.0005806017725011053, P102=0.0003378326634455558, P103=4.2223975262229365e-05, P104=4.5188267143673476e-05, P105=0.00033738110741633086, P106=0.00041126687413030793, P107=0.00012490923500154443, P108=0.00010538268306117879, P109=6.950581533564284e-05, P110=3.491817661458511e-05, P111=3.6818765126797e-05, P112=0.00030627619883629885, P113=0.00018283937483928083, P114=3.669917795422948e-05, P115=7.843163496586025e-05, P116=8.348630711016071e-05, P117=6.146530279410901e-05, P118=0.0002577961506176473, P119=8.399828326957581e-05, P120=7.602462216501711e-05, P121=4.361571743050685e-05, P122=0.00021859973025789843, P123=0.00024165666135332094, P124=1.3408651484054623e-05, P125=2.490020129428651e-05, P126=0.00016649104123651721, P127=0.0001284096339143996, P128=2.4481773046124118e-05, P129=1.526584637492543e-05, P130=3.8410498038276994e-05, P131=1.0329183139634563e-05, P132=7.204030053558055e-06, P133=3.56348530927502e-05, P134=1.2751179423791514e-05, P135=5.160386822494688e-06, P136=4.828057096110025e-05, P137=3.2052992437066545e-05, P138=1.365710137615181e-05, P139=3.425510503719876e-05, P140=1.7307774136926077e-05, P141=8.697189864851963e-06, P142=9.007781544460976e-06, P143=9.472242810502822e-06, P144=1.800474603087533e-05, P145=1.4109548365758497e-05, P146=1.2756801106669184e-05, P147=2.2404260568073294e-05, P148=2.5175218727196384e-05, P149=2.161210079795862e-05, P150=2.0954908497450456e-05, P151=2.056400608247634e-05, P152=1.1447882378199544e-05, P153=1.1044757548929334e-05, P154=1.6107891534032404e-05, P155=1.588773197881392e-05, P156=8.791493372464637e-06, P157=4.634228603456969e-06, P158=3.5453710196096104e-06, P159=5.093619261784547e-06, P160=4.447105101970643e-06, P161=4.245814533128267e-06, P162=5.433442495820966e-06, P163=5.533449934720214e-06, P164=4.818232291939062e-06, P165=4.705134911859395e-06, P166=3.980446634479689e-06, P167=1.5889245728033275e-06, P168=2.1127533217741304e-06, P169=2.253870026744275e-06, P170=3.942594301013696e-06, P171=2.7461245783448524e-06, P172=2.238259910569668e-06, P173=2.000881099260586e-06, P174=2.160615256923836e-06, P175=3.69159614302758e-06, P176=3.2733002276091795e-06, P177=3.998320819229636e-06, P178=2.531267144441936e-06, P179=5.67054432173462e-07),Row( P0=-579.2429309097063, P1=9.025246767765589, P2=-49.778519677147955, P3=-13.373986881344415, P4=-29.649546077659448, P5=-14.779341537195714, P6=-8.439180944894643, P7=-8.466222967907347, P8=2.141029791000822, P9=-0.7429663220710457, P10=4.767453675163234, P11=11.351082225060857, P12=0.6028716090505261, P13=5.976100829240924, P14=-3.1271655207769644, P15=4.210140402171515, P16=0.5242422625895637, P17=1.4515235514208333, P18=0.44218295996911755, P19=-2.837247056412863, P20=0.18648439605564937, P21=-1.1563990915220668, P22=-2.3183688718612045, P23=-1.3146640948123054, P24=2.4548682075809984, P25=-0.27721613409895146, P26=-0.5106519642829616, P27=0.962207423643015, P28=-1.2944018162789241, P29=0.5216617652702432, P30=0.056859861667725616, P31=2.1513422069541717, P32=-0.3408056843453876, P33=-0.4168235915522308, P34=-2.9122164108647475, P35=1.3936567484781217, P36=0.2577740642751104, P37=0.4894854814879681, P38=-1.298066726876518, P39=-0.21851956264404043, P40=0.4709338789071619, P41=0.47978863841892305, P42=0.46900630239413543, P43=0.4429129869687831, P44=0.43035500353659045, P45=0.4503328888852692, P46=0.45786748879895456, P47=0.4560388392083542, P48=0.45880663771938845, P49=0.4633146028963689, P50=0.4907441513055496, P51=0.46586548188764354, P52=2.3424033650827886e-06, P53=3.644067117342673e-06, P54=1.2823732853014476e-06, P55=8.911718502891865e-07, P56=2.348578499832103e-06, P57=3.7777115194148456e-06, P58=2.1736289674418615e-06, P59=1.1430410809622039e-06, P60=1.9571041709880317e-06, P61=5.486019750877639e-06, P62=8.59923295921368e-06, P63=4.519632748261539e-06, P64=1.6999546008255262e-05, P65=0.001044215298483429, P66=0.002980989785136084, P67=0.0026456565951722823, P68=0.002811475852870788, P69=0.009087441591149487, P70=0.013646406688981063, P71=0.017299641013675662, P72=0.024518300927016394, P73=0.02407328011174288, P74=0.0404497266877331, P75=0.05773491335498654, P76=0.05832381935022915, P77=0.08756670812474494, P78=0.10899797005779402, P79=0.17378963641395148, P80=0.238035835762075, P81=0.21111310370743225, P82=0.20428954415932019, P83=0.2739851466545924, P84=0.26024210135531606, P85=0.16293668364833155, P86=0.03705129113232348, P87=0.025492971192305595, P88=0.058908547925813154, P89=0.15086739001650257, P90=0.31549212734907317, P91=0.36537620269174925, P92=0.3096105884739629, P93=0.19605727439481144, P94=0.3219445018709395, P95=0.3638165576757304, P96=0.1589547119919707, P97=0.12088466038096865, P98=0.10360868262789757, P99=0.08549782731694132, P100=0.3111083519956926, P101=0.4617598191462334, P102=0.23355836154643406, P103=0.07371887642617155, P104=0.06841385256216571, P105=0.16671686238348513, P106=0.23382109731333667, P107=0.06127468789011716, P108=0.05002417339834256, P109=0.10132338960812612, P110=0.11153547973179233, P111=0.059132585250082734, P112=0.024584756627852245, P113=0.019028171294803525, P114=0.08762585816576764, P115=0.15476277083745466, P116=0.10659738342011714, P117=0.08231750193242998, P118=0.0351694500244026, P119=0.023475978248445235, P120=0.011336460387230185, P121=0.025648176477548707, P122=0.06639853550028664, P123=0.011476794096248716, P124=0.011399933251925343, P125=0.009974403398251177, P126=0.0196295414763566, P127=0.032700316553853304, P128=0.011975356880183937, P129=0.012525510018322436, P130=0.005850019097674891, P131=0.004655976183356184, P132=0.002493799270738708, P133=0.0025692116663159985, P134=0.0027299385411092532, P135=0.0023648390282096036, P136=0.002378012563163613, P137=0.0059442754526829586, P138=0.006718883448023654, P139=0.006286405133173259, P140=0.010166245853295482, P141=0.004843793030090742, P142=0.002938474810129894, P143=0.003548453457669976, P144=0.002119393425764305, P145=0.003363266489112498, P146=0.004020959458530245, P147=0.0056060556138351906, P148=0.00978463808566388, P149=0.006155324637296765, P150=0.004035523834764774, P151=0.0039523680063301555, P152=0.0026527707219408865, P153=0.0013400487878911466, P154=0.0010956795837997965, P155=0.0011618790406473481, P156=0.0008702970755678231, P157=0.0005291619701890084, P158=0.00025130587519404815, P159=0.00013596967783916964, P160=0.00013179367582974322, P161=0.00012223421760717824, P162=7.452675686601697e-05, P163=9.393941470077648e-05, P164=9.161799020595316e-05, P165=7.355357426332943e-05, P166=0.00010222424462242182, P167=0.00015106288986715827, P168=0.0002460299953871774, P169=0.00018181331760254871, P170=0.00015258159698119196, P171=0.00010774367145928761, P172=6.861194844745466e-05, P173=4.91571514762227e-05, P174=5.7551977529934404e-05, P175=5.617194611754214e-05, P176=5.463404498622233e-05, P177=7.01427250826149e-05, P178=4.067221107676601e-05, P179=6.271455657533971e-06),Row( P0=-732.0575113342422, P1=37.038772804149, P2=-11.805045187465234, P3=-11.881536448405612, P4=-14.67214313119969, P5=-11.168081770211362, P6=-10.77917778611844, P7=-4.126463110112421, P8=-6.9934212808602405, P9=-11.622509616167683, P10=-7.816697281247158, P11=-8.22706186076689, P12=-8.291016571200768, P13=-4.826577307536977, P14=-0.3781279501479842, P15=0.4841624959076427, P16=3.4839348666976746, P17=7.333331499448736, P18=7.44317683526946, P19=4.746005769683338, P20=2.9081604998772255, P21=2.47094902396957, P22=-2.682376859365456, P23=-3.0260842494878903, P24=-3.200018559507611, P25=-3.314854990188749, P26=-4.793686980212726, P27=-0.6799199548945213, P28=-2.0138669723111393, P29=0.614150883178105, P30=-1.5919475773384666, P31=0.9851883888771561, P32=1.0714004171873281, P33=0.4102923698411469, P34=-1.92751864760043, P35=-0.3572495901843527, P36=-1.894224634795096, P37=1.3280299005001814, P38=0.2713259110106305, P39=1.0296168414276323, P40=0.5805750159912152, P41=0.5185399797223414, P42=0.4819514868452642, P43=0.4884993453835264, P44=0.5168704437693088, P45=0.4957458468638582, P46=0.4771521592886083, P47=0.4877429515125122, P48=0.5159495347364246, P49=0.5324540544853045, P50=0.5531728322658404, P51=0.5585406238526037, P52=2.2562790794651198e-07, P53=4.4144768660348434e-07, P54=5.925393815647972e-07, P55=5.745246758728092e-07, P56=2.4762372029183952e-06, P57=9.727127155412222e-06, P58=9.916398799078128e-06, P59=1.422254253899401e-05, P60=6.523026110486794e-05, P61=0.00012930623573095506, P62=0.0004626955860956761, P63=0.0011849429585972092, P64=0.00218000933483747, P65=0.00857707978418632, P66=0.013404220640265342, P67=0.01586502270059231, P68=0.01824493589520173, P69=0.005187116976467894, P70=0.0008178126036333108, P71=0.00015895585758229848, P72=9.946000683391928e-05, P73=0.0001334985192404853, P74=0.00032589171538618565, P75=0.000566769550399972, P76=0.0017952931155990183, P77=0.008838178892741836, P78=0.007604517529425062, P79=0.006132476671756497, P80=0.007902269859901142, P81=0.008643670907884772, P82=0.004462727160308419, P83=0.0024222292010351145, P84=0.00569631455019568, P85=0.011456970999219386, P86=0.004389736515642825, P87=0.0020376022076499323, P88=0.0030603836219985987, P89=0.00784470843671751, P90=0.00751124434535992, P91=0.007836344531022712, P92=0.006552464882725741, P93=0.0038126765726083415, P94=0.0030307591295201993, P95=0.0016981729589426956, P96=0.0008128454235457285, P97=0.0023021931151063425, P98=0.0028916137413286156, P99=0.0006421589595119089, P100=0.000870315379784649, P101=0.001098149012886238, P102=0.0005836512816729107, P103=0.00019948688694531412, P104=0.00034392471038673675, P105=0.0007201724664203682, P106=0.0011025082689289682, P107=0.00048245033889376786, P108=0.0002662683232868011, P109=0.00027094294979311453, P110=0.0003767915595962317, P111=0.0004176120736483999, P112=0.0005129258635846516, P113=0.000358618781405873, P114=0.000552177474798751, P115=0.0003791274941628324, P116=0.00017539229652617795, P117=0.00018301455198335112, P118=0.00010612379702115102, P119=0.00015781870220207012, P120=7.049279083205765e-05, P121=4.692687708487929e-05, P122=6.92920827082805e-05, P123=0.00016343647733904049, P124=9.752496661947729e-05, P125=9.994839378434939e-05, P126=0.000126851153010116, P127=0.0002012076650465559, P128=2.5813816114571927e-05, P129=1.974911270764453e-05, P130=2.019123222164409e-05, P131=1.935961577672579e-05, P132=1.7830154897301156e-05, P133=9.160199964751479e-06, P134=8.755196480546668e-06, P135=1.0866681348248857e-05, P136=1.2390422743851759e-05, P137=1.1261905299217123e-05, P138=1.9323113939989247e-05, P139=2.3815568185328334e-05, P140=1.483529306777151e-05, P141=9.037278165876929e-06, P142=9.121510762648404e-06, P143=9.044564033333612e-06, P144=1.3158629489563184e-05, P145=1.4486484288476524e-05, P146=3.126104282157465e-05, P147=3.109789428971663e-05, P148=4.5100364390090345e-05, P149=3.050627512369515e-05, P150=2.1967475785017224e-05, P151=1.966902525691899e-05, P152=2.068714302255626e-05, P153=1.4626445666546004e-05, P154=1.7996396336563854e-05, P155=2.5016278242064276e-05, P156=3.17368917430856e-05, P157=3.991627431000527e-05, P158=3.680094880890657e-05, P159=4.2261584085885724e-05, P160=2.9043854141874436e-05, P161=2.6952607457256018e-05, P162=3.7750643751733455e-05, P163=3.709770334231283e-05, P164=6.135024108041731e-05, P165=0.00011737402841323261, P166=0.00011090647639960128, P167=7.886495335688542e-05, P168=7.317113409423479e-05, P169=8.558767600636276e-05, P170=9.535210600733272e-05, P171=9.074416800116354e-05, P172=7.36443705437915e-05, P173=6.311621400392341e-05, P174=6.680220681402232e-05, P175=5.3577651728728655e-05, P176=8.589908508059586e-05, P177=0.00011517213913251305, P178=7.361119770227836e-05, P179=7.576204373843384e-06),Row( P0=-572.7138042661755, P1=21.597308673941605, P2=-29.697025653816155, P3=-20.909351273742047, P4=-20.033249972167603, P5=-14.43257576072739, P6=-18.203869515638065, P7=-12.606475098470153, P8=-6.820844143842746, P9=-9.308977572039822, P10=-4.830832867929236, P11=7.088491004359282, P12=3.9755590155146483, P13=0.2593852304991071, P14=-1.6088902438074624, P15=-4.4733573786192915, P16=-6.579884963069881, P17=1.1143190643599508, P18=0.23778638306296496, P19=-0.42402327760160524, P20=-1.9183079775951848, P21=1.032720238430528, P22=-3.72487281771393, P23=0.004135752751081817, P24=-2.7547151847470097, P25=-5.201108571316708, P26=-5.605984085425626, P27=-1.2363090851406404, P28=-1.1997875559373619, P29=1.1470465678261443, P30=-4.241156017869711, P31=0.6626428976680583, P32=0.42450007558065106, P33=-1.3254048274638124, P34=-0.3842229285948517, P35=-1.1317758367192587, P36=-1.1192843959634509, P37=-1.1257171982608378, P38=-3.0255009316846726, P39=-0.6416635061091336, P40=0.5426861408327418, P41=0.5488408611357222, P42=0.5492634794627532, P43=0.523970751049157, P44=0.5643104108773627, P45=0.5905793980324144, P46=0.6436419611027749, P47=0.6020517874465008, P48=0.5836632798243119, P49=0.5502285815343558, P50=0.5354185937967877, P51=0.5383508344506146, P52=2.509428502667602e-07, P53=6.830354160445961e-07, P54=4.023163753219311e-06, P55=5.733027877720233e-06, P56=1.9589416295511424e-05, P57=8.53694711161332e-05, P58=6.283580533834304e-05, P59=2.4931952928009812e-05, P60=4.307421167089913e-05, P61=0.0001338354345740727, P62=0.0005071812531884561, P63=0.0011731421591143188, P64=0.0017244451579802394, P65=0.005425811746902298, P66=0.01660429224910591, P67=0.027401079118683194, P68=0.017413008046456045, P69=0.022159819137739716, P70=0.01976801182829433, P71=0.0109289545158606, P72=0.012147223253171734, P73=0.022096295087431387, P74=0.10221521799298974, P75=0.12349696057603074, P76=0.09652344546609817, P77=0.10556697662312348, P78=0.12928628146414417, P79=0.10905910666308812, P80=0.07158058070864681, P81=0.04862284363852865, P82=0.030156621342050407, P83=0.01374417808052807, P84=0.009512556979431624, P85=0.009125584085995954, P86=0.007373231423583966, P87=0.015338371904382303, P88=0.03479849980186808, P89=0.031984802555604756, P90=0.0298885855558124, P91=0.06693765374677672, P92=0.1129682347496072, P93=0.08074229624115604, P94=0.14520078595509864, P95=0.15546614589566887, P96=0.0692482440353309, P97=0.039674130636902206, P98=0.02798373525942992, P99=0.030397666760151255, P100=0.04676897082864288, P101=0.05156437528018051, P102=0.042613934746558606, P103=0.014061728287337621, P104=0.009868097429800144, P105=0.05745146874735376, P106=0.06788844424713054, P107=0.012721137352041584, P108=0.006237544539054897, P109=0.005350161410908885, P110=0.0091106422467936, P111=0.012121609832488655, P112=0.009672251243967082, P113=0.0241402948381783, P114=0.03096012315477553, P115=0.03578365181461312, P116=0.0279916028828554, P117=0.015557659291911352, P118=0.0047765815677348125, P119=0.002288004197889422, P120=0.0015764086869092427, P121=0.005082624354066304, P122=0.010107085532733345, P123=0.005695137948871881, P124=0.010602799576492466, P125=0.005652271223519909, P126=0.0037577796890715066, P127=0.002722912544138002, P128=0.00301753738757375, P129=0.004832467082606729, P130=0.001982755234836114, P131=0.0004666382589900585, P132=0.00032140439859994083, P133=0.0003644380979057873, P134=0.004173838092891053, P135=0.004158313257447288, P136=0.0004891294061524872, P137=0.0003454344080741122, P138=0.0010086514079503412, P139=0.00266786981624043, P140=0.0009012976665715546, P141=0.0012892892547367153, P142=0.0011887850283677623, P143=0.0013208626575853593, P144=0.000371698281545125, P145=0.0003532304519706139, P146=0.0005369174765633859, P147=0.0005260221379298261, P148=0.0008074531448705599, P149=0.000664537957242755, P150=0.0005620120031652752, P151=0.00026892594972507467, P152=0.00028545455745953056, P153=0.00029091302607707687, P154=0.0003321048317777998, P155=0.0004392499289066576, P156=0.00037468516097892, P157=0.00043343932857878797, P158=0.0005353839814308595, P159=0.00031721978544914457, P160=0.00026390817664018515, P161=0.00038119438521669313, P162=0.0004199739075584502, P163=0.0005072864487631497, P164=0.0004648611850128703, P165=0.00038358048708972084, P166=0.0004200988281660045, P167=0.00035308281929023294, P168=0.0002927320819685306, P169=0.00034143788628450633, P170=0.00046939410562491055, P171=0.0003992078494170511, P172=0.00032471354653991605, P173=0.00037645869530586213, P174=0.000559880402444001, P175=0.0005433264557393255, P176=0.0004984379103940923, P177=0.0005940234795784853, P178=0.00033597367655539785, P179=3.620639115050274e-05),Row( P0=-874.8955183272062, P1=31.41561694468573, P2=-7.944544865344194, P3=-4.131236608692566, P4=-18.19070057768747, P5=-10.156371789165984, P6=-11.001046653793164, P7=-7.449785906332804, P8=-5.9969768086056785, P9=-9.376385633468756, P10=-6.293306357696538, P11=-0.7462110387910494, P12=-2.7854171395151472, P13=-1.766921069406124, P14=1.579022288805401, P15=0.19673338765898413, P16=-2.9065698748209012, P17=6.305658378972165, P18=3.99228663871799, P19=8.688757812677014, P20=9.134890234427742, P21=8.187556160718728, P22=2.104802381761161, P23=-0.6051518746184814, P24=-1.7195673282384571, P25=-2.1285283341698245, P26=-2.361344011542305, P27=0.6299041654435712, P28=-2.1319646039820954, P29=-0.6119232573832196, P30=-2.218432762139723, P31=-2.5381211154405596, P32=0.21432932927244472, P33=0.011159479320548851, P34=-1.2930571416981178, P35=-0.5877703382993196, P36=-2.532680174039086, P37=2.1854929153834894, P38=2.4292995224382916, P39=3.554057655287796, P40=0.4512390543132246, P41=0.41813283483398794, P42=0.39115668451103836, P43=0.43942234265837976, P44=0.4705837053553352, P45=0.4404459897952729, P46=0.43679940797684336, P47=0.47096130317977564, P48=0.5624902874986925, P49=0.5963071140415741, P50=0.5398296696066933, P51=0.4934764176156388, P52=2.0120818599380625e-07, P53=2.64529697278113e-07, P54=1.6813638051630284e-07, P55=3.1669015004362615e-08, P56=1.0748862581593898e-07, P57=2.918466310507035e-07, P58=4.2799935984676695e-07, P59=2.6625207244501294e-07, P60=3.8166200252985675e-07, P61=2.310255316261056e-06, P62=3.749959972053632e-05, P63=0.0004155066001767368, P64=0.001432219352691168, P65=0.0014984971113412718, P66=0.0009378582304467556, P67=0.0005403454224400694, P68=0.00017460778773198624, P69=0.0002395440537113566, P70=0.00025553269916632194, P71=0.00016561204210171392, P72=0.00027808158278883375, P73=0.00026736068188571956, P74=0.00017212157668921856, P75=0.00010749261449433602, P76=0.00013086147036303408, P77=0.0006229010665246463, P78=0.0006834143350191291, P79=0.0003287615304162758, P80=0.00013767275202469323, P81=7.417655032362001e-05, P82=3.651867247778637e-05, P83=2.3262550262011485e-05, P84=2.6130809951621064e-05, P85=3.374112797058816e-05, P86=2.7744042185308968e-05, P87=4.7189525121137246e-05, P88=0.00010689889675408743, P89=0.00018179822428977704, P90=0.00014779107030392976, P91=9.815302834002788e-05, P92=3.876001111507219e-05, P93=4.235991068324117e-05, P94=7.167251826260444e-05, P95=0.0001024085652406823, P96=4.5995813475179466e-05, P97=3.68622588222335e-05, P98=5.9512987499711365e-05, P99=4.0374017683693306e-05, P100=1.2380452708398788e-05, P101=1.0321387126885322e-05, P102=2.1365376341286893e-05, P103=1.113904847479061e-05, P104=2.233314176806167e-05, P105=1.2600833732937094e-05, P106=9.837551637346236e-06, P107=4.492815271251871e-06, P108=1.555450494526122e-06, P109=3.971348388322068e-06, P110=7.966223172978913e-06, P111=5.592243748721026e-06, P112=3.1752574620105735e-06, P113=8.860796664633503e-07, P114=4.633880615533304e-06, P115=1.7490058474718408e-05, P116=1.1511614328967617e-05, P117=2.9003411061633997e-06, P118=1.832870860308886e-06, P119=6.102196510521215e-06, P120=5.965504265037708e-06, P121=2.9057345512063877e-06, P122=1.7609518074711832e-06, P123=6.4557455316466495e-06, P124=4.8894282905871625e-06, P125=3.6464682699637415e-06, P126=4.940666766222089e-06, P127=6.31906647785071e-06, P128=3.5069453149059764e-06, P129=1.4744460152385147e-06, P130=1.4728792621598576e-06, P131=1.0031862793096493e-06, P132=4.4842100972980067e-07, P133=6.095552027758992e-07, P134=3.6229376852145467e-07, P135=5.082265888973355e-07, P136=4.6154233012723247e-07, P137=6.844518367071468e-07, P138=1.7850879585028483e-06, P139=2.9788864645386075e-06, P140=3.6822297974869627e-06, P141=2.164077796055826e-06, P142=2.7505897029712115e-06, P143=1.984963342854147e-06, P144=3.550435673762329e-06, P145=5.012162129623078e-06, P146=2.7540171315570608e-06, P147=1.7963421842408288e-06, P148=1.7711525047935995e-06, P149=1.6755461309799726e-06, P150=9.765533259562192e-07, P151=6.073871642009972e-07, P152=1.217133214268718e-06, P153=2.7559837568156723e-06, P154=4.076466208748168e-06, P155=3.7756671872127452e-06, P156=3.944420659899329e-06, P157=2.273457230682685e-06, P158=1.8426415436771213e-06, P159=3.379256058382819e-06, P160=3.4127661054328723e-06, P161=4.689912651087959e-06, P162=4.21420171455531e-06, P163=3.011965437795958e-06, P164=1.690541347094508e-06, P165=1.535910214103945e-06, P166=1.983445002246376e-06, P167=1.8193026848085305e-06, P168=1.6223055789937221e-06, P169=2.539795633263127e-06, P170=2.7098058845482824e-06, P171=2.997388095682951e-06, P172=3.691624598375985e-06, P173=2.878777720606489e-06, P174=2.7474179132092707e-06, P175=2.851949607820155e-06, P176=3.5787406138464718e-06, P177=3.4030986122056292e-06, P178=1.3667063074527163e-06, P179=1.4158114782010143e-07),Row( P0=-592.7312676974843, P1=7.756623451962754, P2=-23.863373005041897, P3=-23.415402176302848, P4=-19.008075628655497, P5=-16.194416407292042, P6=-11.905548701064136, P7=-5.951068106823555, P8=-5.09773893250607, P9=-4.227107845275651, P10=-0.700715768066851, P11=7.224067263612182, P12=0.4793887277460927, P13=0.25431815173978645, P14=1.8510433174048235, P15=-1.6213547449883883, P16=-1.83541847759963, P17=2.6826309045744954, P18=0.7596872464671713, P19=1.3668329060872757, P20=1.2242109200095441, P21=1.3606736020711017, P22=-0.4276256882962533, P23=-0.3196976177239088, P24=-0.1284429683492099, P25=-1.8239162620897293, P26=-2.5578162898889993, P27=-2.6013979283115503, P28=-0.153901715723227, P29=0.5880741130724787, P30=-0.8023523519627926, P31=0.5082587059110083, P32=0.8451944774774525, P33=0.49584655040165276, P34=0.009454634124997155, P35=-0.8895081080046802, P36=-1.4214337263332735, P37=-0.5099963293903362, P38=-1.5035937668880428, P39=-1.0549246790369966, P40=0.5910361696820929, P41=0.561120653120842, P42=0.5240513925844767, P43=0.5438308944021412, P44=0.5909518052735824, P45=0.577633033497699, P46=0.6028540503716828, P47=0.5968670694816761, P48=0.5526879408311769, P49=0.5710608668190678, P50=0.5764619801807294, P51=0.5676253695484134, P52=9.601225750986287e-08, P53=1.1359862029007236e-07, P54=5.587283034099137e-07, P55=5.234681054405299e-06, P56=1.1684918979560505e-05, P57=6.893095279795255e-06, P58=1.347042979342707e-06, P59=1.631505090637239e-06, P60=9.173095098092819e-06, P61=2.4001424589182927e-05, P62=0.00033149505561934573, P63=0.0007419290177206458, P64=0.0004633968302173485, P65=0.0015811079297559452, P66=0.005076689110697717, P67=0.008709499031881594, P68=0.006020431035088602, P69=0.010984906870045708, P70=0.014936011006395277, P71=0.010469942643407422, P72=0.018583773829718434, P73=0.02723066716524369, P74=0.12666464574708103, P75=0.2119097790453032, P76=0.09694260016918207, P77=0.07042615799979396, P78=0.08094685430816409, P79=0.08217438646255006, P80=0.05383707669079959, P81=0.05036469517493737, P82=0.056783286153830284, P83=0.09274273435106525, P84=0.09236691896078736, P85=0.058057694878680455, P86=0.01716230933683043, P87=0.06717547296623895, P88=0.34275276292170476, P89=0.3447461969346089, P90=0.2163952943667666, P91=0.26592529985703994, P92=0.1673545906002507, P93=0.09774216757063746, P94=0.08976552066463733, P95=0.20337471758712827, P96=0.07556613500729745, P97=0.042890727886412974, P98=0.032391557630234415, P99=0.03463487570832897, P100=0.05380054125910239, P101=0.04061069843023071, P102=0.016079647464045246, P103=0.00866283875510985, P104=0.0173113817694705, P105=0.030786330355248737, P106=0.03104255815059963, P107=0.02216039613196925, P108=0.012818287242618267, P109=0.011396228562808239, P110=0.008486029875675743, P111=0.011211291007375302, P112=0.016675143139261668, P113=0.008557544143769687, P114=0.02818860261892183, P115=0.041046166086070146, P116=0.012220292488921779, P117=0.007285273837697581, P118=0.002725761244278502, P119=0.0010453805339773997, P120=0.0036806122651217533, P121=0.014540175410227655, P122=0.006964869062603378, P123=0.0034463822767502763, P124=0.004115632466827605, P125=0.004783478786246136, P126=0.008799300907161084, P127=0.02224196374425688, P128=0.06074852273749994, P129=0.006305331814358748, P130=0.0009724742876214924, P131=0.00026520800215161543, P132=0.0002657934905234113, P133=0.0016313397424547597, P134=0.0012669980818757625, P135=0.0008845011647485221, P136=0.00020854526337910426, P137=0.0004462146765676951, P138=0.0011998399308221304, P139=0.0022142283309782643, P140=0.0008419978637232668, P141=0.0010772145691789, P142=0.002061777534702628, P143=0.0011112224306280129, P144=0.00043886466153771303, P145=0.0005114001117776611, P146=0.0004874491836628805, P147=0.0007239620887904176, P148=0.0005876513408107303, P149=0.0003399431013141053, P150=0.00032426458112078143, P151=0.0001896862903354781, P152=0.000206947339443785, P153=7.694388629426798e-05, P154=8.184573026087635e-05, P155=0.00013910667105594086, P156=0.0001747818441348464, P157=0.00016604934826546595, P158=0.00018603264626470355, P159=0.00020310199416999814, P160=0.00023937495209248942, P161=0.00028176693515898955, P162=0.00023346162677280783, P163=0.00018696739020912295, P164=0.00021166582970321888, P165=0.0001874129056516327, P166=0.00017353380408381954, P167=0.00011821221502322297, P168=0.00012513265672130296, P169=0.0001452824483655749, P170=0.00017532271558187354, P171=0.0002366235638226065, P172=0.00022242655514595876, P173=0.00015757332460932308, P174=0.00018276674506025948, P175=0.0002164509064861557, P176=0.00024204485054462213, P177=0.0002803298745460824, P178=0.00023118517027281816, P179=3.0608121775346286e-05),Row( P0=-814.4947911296161, P1=33.67115554739631, P2=-7.714391359481899, P3=-5.621067682068606, P4=-15.3280907377194, P5=-10.849177090470182, P6=-7.915493347297198, P7=-4.719766305842385, P8=-4.559196902079286, P9=-9.36275935502753, P10=-5.090341146261645, P11=-4.352957514752882, P12=-3.2300292819629783, P13=-4.587960956846419, P14=1.1874747604271016, P15=2.164726642489737, P16=1.742789930701002, P17=10.69960942928813, P18=7.551089935100847, P19=6.717549925705636, P20=5.157183952260634, P21=5.184581286203919, P22=-0.9254091419103376, P23=-1.8272244915133187, P24=-2.337393297652981, P25=-4.645701976116512, P26=-4.535211509803357, P27=-0.8062583893864851, P28=-3.610400982318524, P29=0.41854556496461215, P30=-2.5743498718735767, P31=-1.623572669618311, P32=-0.5946594101816182, P33=0.002240316725089169, P34=0.0762775772098289, P35=-0.8041635213564522, P36=-1.6964336887986848, P37=1.92198523682701, P38=1.114472444379991, P39=3.649157255424826, P40=0.5017539641886437, P41=0.4798672074959845, P42=0.41154615037101827, P43=0.41831923936741083, P44=0.44147404630920856, P45=0.42488313475264544, P46=0.4243135546611771, P47=0.4581516497872416, P48=0.5182679626091888, P49=0.5612764292574016, P50=0.5276082622063623, P51=0.5013853363676427, P52=3.129276191014032e-07, P53=8.540668019073285e-07, P54=8.040179566981544e-07, P55=1.5853596714602832e-07, P56=1.0080935926384343e-06, P57=5.490208750675405e-06, P58=5.040202704862153e-06, P59=1.952164503333338e-06, P60=9.641992291044856e-07, P61=3.4275836621775536e-06, P62=5.51885854188932e-05, P63=0.0005201351410557621, P64=0.0018255995494568784, P65=0.004109586474439326, P66=0.004315260093519161, P67=0.005711550933797684, P68=0.003062358349477135, P69=0.0003813982337320454, P70=8.055842418804505e-05, P71=2.7472195588069833e-05, P72=3.0353369782183917e-05, P73=2.42302215676832e-05, P74=2.7798872827550728e-05, P75=6.004192485589607e-05, P76=0.000259977549034971, P77=0.0009842981249217296, P78=0.0018197103763320029, P79=0.001232764675971747, P80=0.0006893806112637223, P81=0.00077001491500771, P82=0.0009810375264916501, P83=0.0007317232969293538, P84=0.0008162258958698442, P85=0.0008436449882104558, P86=0.00011209081219791947, P87=3.1539007040431434e-05, P88=0.00016852556014554325, P89=0.0004074181334796631, P90=0.0010415914919075582, P91=0.0007110849804832421, P92=0.00011685667213282235, P93=5.978709193037657e-05, P94=9.985671106009203e-05, P95=0.00020951300767476082, P96=0.000343670641121806, P97=0.00044903925926736196, P98=0.00013836515006935307, P99=5.787099805048038e-05, P100=5.975646167065638e-05, P101=3.3820900025742924e-05, P102=1.2021604420615865e-05, P103=3.132398056013356e-06, P104=1.4908215891032521e-05, P105=9.422485389878917e-05, P106=0.00014483507009681003, P107=2.035182570039108e-05, P108=2.3380369583751127e-06, P109=4.198250645567786e-06, P110=1.2560916186046864e-05, P111=4.6027711727826284e-05, P112=6.145812766899845e-05, P113=1.0132329011772207e-05, P114=1.2292486342194661e-05, P115=1.9058073763499208e-05, P116=1.710040705553224e-05, P117=1.8435389967105373e-05, P118=2.305026343183424e-05, P119=1.335108469486219e-05, P120=1.4692260796973724e-05, P121=1.962565810551746e-05, P122=1.3166990001850568e-05, P123=2.8951013457378073e-05, P124=2.006623322847489e-05, P125=1.3737527810573485e-05, P126=1.3653036647579876e-05, P127=2.1965520631226702e-05, P128=1.778076858484515e-05, P129=6.737451194813952e-06, P130=4.705735519256512e-06, P131=2.0642196971660924e-06, P132=1.3907502668506569e-06, P133=1.2549419295500833e-06, P134=1.688929575504811e-06, P135=2.4239927335613294e-06, P136=2.5370929386502916e-06, P137=4.818058617730081e-06, P138=5.303209675820426e-06, P139=7.985651030975583e-06, P140=1.0912364875802281e-05, P141=7.765304152658662e-06, P142=6.696103491822141e-06, P143=5.107598280743403e-06, P144=5.5684825900751656e-06, P145=5.51928430706723e-06, P146=3.2449323510373857e-06, P147=4.0426739585370266e-06, P148=4.966957137801364e-06, P149=5.818138144171861e-06, P150=5.6781626150237116e-06, P151=3.7459954256371425e-06, P152=2.7062192874981822e-06, P153=3.7888321956454608e-06, P154=1.0064227432366426e-05, P155=9.616341920290088e-06, P156=1.224785634116628e-05, P157=1.3606707967027164e-05, P158=9.67082482348712e-06, P159=8.623909358701569e-06, P160=6.3500211062780406e-06, P161=5.016161264333941e-06, P162=6.76374465456288e-06, P163=1.1756070004981091e-05, P164=8.313125312259506e-06, P165=9.419913235142541e-06, P166=1.3524840267242345e-05, P167=1.1078475675640112e-05, P168=6.190890177589458e-06, P169=1.1616348534393622e-05, P170=1.4378243713666254e-05, P171=1.3371675533032648e-05, P172=1.4334597870621424e-05, P173=1.0058946905269343e-05, P174=9.095604393172786e-06, P175=8.957007663914245e-06, P176=9.228332175154477e-06, P177=1.0481249207670703e-05, P178=6.62965783200582e-06, P179=7.471931665251882e-07),Row( P0=-361.23406913322697, P1=-0.5304687702184765, P2=-58.581321371512665, P3=-17.444507008667003, P4=-22.577577470692315, P5=-3.221320268108345, P6=0.992687927679175, P7=7.936486748236596, P8=16.99772676845544, P9=0.8655273732497111, P10=4.548395010642974, P11=7.863439507374867, P12=-3.7141824636457708, P13=9.391598873837731, P14=0.15135179688000322, P15=3.4180825982231147, P16=-11.639420030694115, P17=4.246700728561602, P18=2.5571045336998606, P19=1.7121934382177049, P20=4.166383498337297, P21=2.053439776529908, P22=2.308487565623354, P23=4.1947322231296775, P24=-0.5753191137914763, P25=-1.5231150533845714, P26=-1.3927643433699743, P27=2.7807457295979963, P28=-2.900376725930591, P29=3.7666507822089015, P30=-1.5713869784135912, P31=0.004554106203490309, P32=-0.42463372869815447, P33=0.7833594097118732, P34=-1.2432761238487637, P35=2.0114667698073307, P36=-2.701885820316612, P37=3.9725626105631995, P38=-2.078924162524119, P39=-1.1285391933067783, P40=0.6595211211070967, P41=0.6545470401378944, P42=0.604894266910795, P43=0.586752447550196, P44=0.5971532184159255, P45=0.5871566965149935, P46=0.5817408925936646, P47=0.5459950721015167, P48=0.5733120030452439, P49=0.6015694571726077, P50=0.5574200209101527, P51=0.577605974136306, P52=0.031228743096015178, P53=0.05203138584555334, P54=0.03939261758562784, P55=0.009903720305615659, P56=0.0013987308605049203, P57=0.00041066197259112704, P58=0.00016281708675763365, P59=0.0001487926705493256, P60=0.0001350844341240279, P61=0.00012747192896383136, P62=0.00015460029208578526, P63=0.00021092617461650675, P64=0.000162544943918692, P65=0.00022264856718553035, P66=0.000245007867744495, P67=0.0002572189354143491, P68=0.00025883329203263194, P69=0.000468134517146896, P70=0.0006426933652914122, P71=0.002131606212457891, P72=0.007771056436418262, P73=0.011485033609056165, P74=0.018119761982134587, P75=0.02878094115438597, P76=0.032561495339475624, P77=0.06919886707535297, P78=0.14079538679040698, P79=0.2982840259636667, P80=0.26840273717263513, P81=0.19592875841301513, P82=0.1831897683372739, P83=0.28448905392807605, P84=0.4495731840009176, P85=0.684519934500341, P86=0.8372574777727743, P87=1.576180430334842, P88=1.5788060766981789, P89=1.471379648286722, P90=1.96321559549589, P91=2.0354175622957196, P92=1.1289525513644396, P93=1.7128222336858947, P94=4.341436006619991, P95=3.1669852477883484, P96=1.549235291517572, P97=3.506816496913609, P98=2.860815924052341, P99=3.120968582595466, P100=9.725051690290618, P101=5.070419959816132, P102=1.017996684831472, P103=1.8081677447476021, P104=1.9055118294139795, P105=6.782197661675137, P106=6.887891740238292, P107=4.129805565939719, P108=1.85968759159372, P109=1.3350159155932704, P110=1.0656010162604101, P111=0.8867452648730064, P112=0.6461003621452021, P113=5.586687666025762, P114=2.396752728744869, P115=1.8789414446042991, P116=1.0423367026052428, P117=1.155923785425931, P118=1.305989900365713, P119=1.187020097940962, P120=0.4744620721670497, P121=0.1330978476780024, P122=0.16305411854108035, P123=0.1248659659467675, P124=0.1537510780110832, P125=0.10916773549122352, P126=0.210236186825774, P127=0.3967268232856406, P128=0.1826587737233094, P129=0.11850487331716561, P130=0.1429823288100312, P131=0.06251077243712382, P132=0.03674636288335935, P133=0.02480902669875696, P134=0.024452765805436705, P135=0.12414220770498553, P136=0.14398562370854115, P137=0.05355271758474092, P138=0.047875781761496365, P139=0.041414326660783554, P140=0.03851732253539237, P141=0.038707218726226976, P142=0.02278327501494242, P143=0.022569287238355138, P144=0.010913681106438434, P145=0.012427940499388971, P146=0.011906347090189163, P147=0.016853364141417224, P148=0.022841398936168215, P149=0.022677701184881452, P150=0.012691233250858, P151=0.015341629579021387, P152=0.014965359248842802, P153=0.01592776746596152, P154=0.013467205036626087, P155=0.010956395892845567, P156=0.009078223226931933, P157=0.006787357417754553, P158=0.0028664362177784224, P159=0.0015163736545065797, P160=0.0012927894023156811, P161=0.0009548567210123237, P162=0.0006279603580482788, P163=0.0005933373793812727, P164=0.0006193589379087196, P165=0.0007088324281101233, P166=0.0010513275493174872, P167=0.0011111492008296202, P168=0.0019413455441862207, P169=0.002297663018031381, P170=0.0023872315224147895, P171=0.0014769641149368963, P172=0.0013279303121636827, P173=0.0013004451035247896, P174=0.001778541228481595, P175=0.0022028356357574044, P176=0.0019512015866326353, P177=0.0018901223699763253, P178=0.00080047020838674, P179=0.00010970033193509831),Row( P0=-672.3368792731472, P1=32.16819121322062, P2=-22.371764046320457, P3=-10.396661455550596, P4=-18.724454867088863, P5=-11.32859401199789, P6=-9.541385722165998, P7=-7.92280335854236, P8=-6.728467063832229, P9=-10.805965197626628, P10=-8.632809147575646, P11=-1.7399071386589207, P12=-3.523146887712759, P13=0.17812229740093652, P14=1.2929038910593187, P15=-0.39244144542190834, P16=-0.18319629791359532, P17=3.271115287147404, P18=4.228853154683168, P19=2.1071307464204154, P20=-0.11442481720122093, P21=0.6779299919552235, P22=-4.034457528625981, P23=-1.6337664075820522, P24=-2.101070727836522, P25=-2.8190050658552113, P26=-2.6822624495885963, P27=0.20952802995709874, P28=-2.068668534385384, P29=-0.3473732104412913, P30=-0.3731570736822373, P31=0.4103244401658179, P32=0.5472293348311146, P33=-0.5148814061037933, P34=0.23304497860907988, P35=0.9055389844205967, P36=-2.352492631194725, P37=-0.8421767925705564, P38=-0.6061832037785421, P39=1.8138263063365692, P40=0.5277218425147154, P41=0.4935155915697294, P42=0.4925932935874185, P43=0.49897818225295526, P44=0.5075412707689532, P45=0.4795707148613333, P46=0.4952768021055822, P47=0.4988274328421768, P48=0.4952612955649114, P49=0.4738298794852971, P50=0.4806140923322228, P51=0.49660860667225487, P52=2.9596033111353587e-06, P53=2.024219197077072e-06, P54=1.3093675079115115e-06, P55=3.873043216136849e-06, P56=2.2371044502550763e-05, P57=4.853528906216433e-05, P58=2.0807275042519348e-05, P59=1.3923164524125692e-05, P60=4.064896011040595e-05, P61=0.0001622462301297358, P62=0.0009214002970913997, P63=0.0022065638970369702, P64=0.0038102397342495153, P65=0.010444406227195491, P66=0.014360947159986247, P67=0.015469688986567828, P68=0.020195068521627616, P69=0.03473977455544834, P70=0.030080413308305087, P71=0.013309626580340213, P72=0.0064093560866842255, P73=0.0028752939531040485, P74=0.0034626528902999623, P75=0.005944148024954178, P76=0.009082993506904212, P77=0.016160942675544823, P78=0.01878360347596793, P79=0.01879425555028711, P80=0.02166317724459319, P81=0.01473681135453997, P82=0.00790173628620288, P83=0.006477133447686323, P84=0.01141664195932337, P85=0.023639672747699104, P86=0.009123827870370046, P87=0.0033563049313990083, P88=0.0050213299250887475, P89=0.008153941652368343, P90=0.010025460398623749, P91=0.013402995815514945, P92=0.014194050490999808, P93=0.012446430108545497, P94=0.016169949370985037, P95=0.02374479847894682, P96=0.012063408305301478, P97=0.007946476556432033, P98=0.005726926523231508, P99=0.0025082843666229143, P100=0.003019352305523526, P101=0.0047640123187528945, P102=0.004554834598616251, P103=0.002527950760757457, P104=0.004586808913893311, P105=0.006920217137030032, P106=0.00862232848175169, P107=0.004143098637359493, P108=0.0034475202465743464, P109=0.0030006473584387387, P110=0.003837328261952364, P111=0.003521438352655071, P112=0.0032873226610811756, P113=0.0014500802097733496, P114=0.0019024401164383436, P115=0.002343172697126757, P116=0.0015163913152546405, P117=0.0027266236171111327, P118=0.0014434927789625702, P119=0.0004731588117775904, P120=0.0008872709296429621, P121=0.0012456211975823512, P122=0.00023861879673122753, P123=0.0004254553575663112, P124=0.0009809364726474769, P125=0.002468622220261575, P126=0.0020808851100876653, P127=0.0009138350285678979, P128=0.0003232849028729162, P129=0.00018505423819175707, P130=0.0004970978002632993, P131=0.0008914569079456141, P132=0.00021242010178799077, P133=0.0001551511649448272, P134=8.187403233372452e-05, P135=0.00016552157232640634, P136=0.00014591429526826134, P137=0.00016630584741745102, P138=0.00017695877167450997, P139=0.000204829361554401, P140=0.0002997821356479885, P141=0.00044467912212253655, P142=0.0004083532929253111, P143=0.0004271982326176033, P144=0.0002951686715791928, P145=0.00027746654854214116, P146=0.0003090190264537143, P147=0.00025565081314787483, P148=0.00018989862798804918, P149=0.00023393970341054927, P150=0.00019293290647082339, P151=0.00019350618272995435, P152=0.00015752432411396335, P153=0.00012494731580579954, P154=0.00011776905876413596, P155=0.0001413984477994235, P156=0.00015015049654786805, P157=0.0001285290649884272, P158=0.00010894536064647531, P159=8.990560158458715e-05, P160=7.713404317865378e-05, P161=9.526740517542851e-05, P162=0.00012296063393165744, P163=0.00016879672033370528, P164=0.000172262413496135, P165=0.00012594019344689363, P166=0.00011826564125327857, P167=8.70933471554066e-05, P168=8.138754935380921e-05, P169=7.81449560336609e-05, P170=5.8162847382847264e-05, P171=5.914873328539044e-05, P172=3.494368822141471e-05, P173=1.7888980397919375e-05, P174=1.8001053962394177e-05, P175=2.197541284017013e-05, P176=2.9524630843323923e-05, P177=3.790176667892119e-05, P178=2.589200446894204e-05, P179=3.380864128609117e-06),Row( P0=-711.7465411240045, P1=32.30614292489761, P2=-13.503996038008614, P3=-14.97626087512508, P4=-19.035908494943737, P5=-16.01542637801405, P6=-11.176686880855984, P7=-5.145012449383153, P8=-11.34148240981758, P9=-14.641200561169116, P10=-10.316986063922727, P11=-9.025701706626624, P12=-8.3232699218378, P13=-2.219088721161275, P14=0.3084177669496202, P15=2.674113532397706, P16=2.244038946800855, P17=8.83850813135617, P18=7.408223596152923, P19=5.331521347993988, P20=2.9166363892544473, P21=2.510517960902559, P22=-1.4738696103656412, P23=-2.1857970025310625, P24=-0.4645668547011764, P25=-0.5674647465983932, P26=-2.8142684613745903, P27=-2.873898700821836, P28=-4.428418978670601, P29=0.8462836973966072, P30=0.0855263227187158, P31=0.9558634493019955, P32=-1.137209446802479, P33=-0.6048242001923875, P34=-0.8838987347624918, P35=0.8122799120914644, P36=-1.5751266263610717, P37=-0.6969478313952958, P38=-1.1263753996453287, P39=0.6288738943744265, P40=0.5904607736989455, P41=0.5529806198067077, P42=0.5178593517852235, P43=0.4930008557439022, P44=0.518649214570046, P45=0.5187728969788132, P46=0.49796467388610605, P47=0.5020165617949995, P48=0.4437593614757079, P49=0.46651444139114917, P50=0.5187816765973555, P51=0.5702673344402472, P52=1.4650041657553207e-07, P53=1.875834541705605e-07, P54=1.9183985889814351e-07, P55=1.9286698928089038e-07, P56=2.234732073796205e-06, P57=8.180544199132344e-06, P58=6.040005658227455e-06, P59=8.048814388046874e-06, P60=3.316067374278643e-05, P61=0.00023358319537491578, P62=0.002206102862161685, P63=0.0037248686346350204, P64=0.002244358265451175, P65=0.005403691486683043, P66=0.016582105370507288, P67=0.028372485756151745, P68=0.013497528262572174, P69=0.001970227584314873, P70=0.0025365644682816928, P71=0.0013032384508665962, P72=0.0003464709275066255, P73=0.0008851684107117303, P74=0.0025511677686250304, P75=0.0038866907843085746, P76=0.0024855457515148113, P77=0.0034505631113571273, P78=0.006068986705424013, P79=0.00538395930136892, P80=0.004940723797034624, P81=0.005077527375763319, P82=0.005788051727322339, P83=0.0072112046259983804, P84=0.009804896388610871, P85=0.008970731381114937, P86=0.005508939359716239, P87=0.0037725387361418144, P88=0.0029258988075276196, P89=0.003980945634777383, P90=0.005108129731297445, P91=0.006819834963956035, P92=0.00373321373229066, P93=0.0024148464999605994, P94=0.0039867317222325544, P95=0.007630831779889749, P96=0.0020145803022485944, P97=0.000534694596471634, P98=0.00022382466714685598, P99=0.00016563897217239734, P100=0.0001881675648482046, P101=0.00033080597537677245, P102=0.00034601765368594674, P103=0.0002803145264393356, P104=0.0007498877524538423, P105=0.0002549286674736658, P106=0.00027318884620424096, P107=0.00017407032146501606, P108=0.0001630103029627481, P109=0.00018816421652216402, P110=0.0003269741440300904, P111=0.00027264832939650314, P112=0.0002443554247186855, P113=0.00017051564779334115, P114=9.587077116154155e-05, P115=0.0001335832386034227, P116=9.450143557917181e-05, P117=6.465786701488748e-05, P118=2.881580291353429e-05, P119=2.079312603948662e-05, P120=4.7159871126566376e-05, P121=0.00011367837519583911, P122=4.470471181740163e-05, P123=7.286847412283147e-05, P124=0.00019081317559936105, P125=0.0001538928846483358, P126=7.142539736188306e-05, P127=7.953962427606769e-05, P128=2.4684508622984444e-05, P129=2.9827805219791622e-05, P130=2.705580733894933e-05, P131=1.53683847344588e-05, P132=1.6682859914719195e-05, P133=2.8381427341791428e-05, P134=1.620348760554272e-05, P135=1.863820378771427e-05, P136=1.4930082168932473e-05, P137=1.64510654476803e-05, P138=2.3033338215514163e-05, P139=2.271387654323023e-05, P140=3.2345843912962464e-05, P141=1.9702427464312416e-05, P142=1.9635994651794235e-05, P143=1.597272215691915e-05, P144=1.248078279310469e-05, P145=1.6541698046532787e-05, P146=2.9007832054337283e-05, P147=2.967996839057872e-05, P148=6.184641705293482e-05, P149=6.469772346270587e-05, P150=6.525985463445294e-05, P151=5.269407625917201e-05, P152=3.617922132278017e-05, P153=4.4003653888521026e-05, P154=7.877419639280287e-05, P155=4.988220853522018e-05, P156=5.132963195750826e-05, P157=0.00010648984465506885, P158=7.164713957429039e-05, P159=3.8586790040669115e-05, P160=3.5517751275145595e-05, P161=3.835833458714314e-05, P162=2.6645403137284517e-05, P163=3.86340016744234e-05, P164=5.075327597383316e-05, P165=7.093114614050056e-05, P166=9.922252132022144e-05, P167=8.609005379447006e-05, P168=9.733408622133583e-05, P169=0.00011060899136680566, P170=9.618448570772378e-05, P171=8.240415493176907e-05, P172=7.891933285044546e-05, P173=7.268097487236665e-05, P174=5.911347112770944e-05, P175=5.468143615215964e-05, P176=5.023437913603096e-05, P177=7.19686784878967e-05, P178=4.445874592130045e-05, P179=5.3452546783273854e-06),Row( P0=-705.8431098246252, P1=17.37869712105456, P2=-27.271944212652166, P3=-11.28619128124593, P4=-19.79412403308202, P5=-16.568299031310687, P6=-8.829584866612173, P7=-5.207520795143883, P8=2.245293123455132, P9=-4.501664970506574, P10=2.552940753226223, P11=4.486605898453254, P12=5.101802275849867, P13=5.010485534525212, P14=-3.6301710371142297, P15=0.9864775067715513, P16=-7.910800255743929, P17=2.592847405355107, P18=1.7384269526071439, P19=0.5077693028416779, P20=1.8621227276377106, P21=2.71233686409961, P22=3.7246947873218157, P23=3.199874850998237, P24=1.3346117153921668, P25=-1.1856807005195766, P26=-1.8071347223964425, P27=0.4567346425434079, P28=-1.2150489557809654, P29=-1.2353488966858535, P30=-1.5432800598261882, P31=-0.20159257045580786, P32=0.17959196647043446, P33=0.672392825273169, P34=-2.7651109575684365, P35=-0.3758830572505114, P36=-1.8427780295473812, P37=2.7038550395422605, P38=0.06832966014911838, P39=0.017733638191553498, P40=0.4858707108372813, P41=0.47533431158639367, P42=0.4828091664393602, P43=0.5242089527869597, P44=0.5344101333641511, P45=0.5544603637483617, P46=0.5678937677340622, P47=0.5066089947029965, P48=0.5237493898745615, P49=0.5280275315497371, P50=0.478089292420368, P51=0.4473100473186743, P52=3.6667776092919776e-07, P53=1.2859476199236368e-06, P54=6.441204353435926e-07, P55=1.663400788598846e-07, P56=8.444837947893225e-07, P57=1.1665996151639925e-06, P58=6.816071404638078e-07, P59=5.27997964712725e-07, P60=3.8444103820486796e-06, P61=0.0001472496076940117, P62=0.0011276292291998171, P63=0.0021733609270774526, P64=0.00412000761557608, P65=0.008458259891857611, P66=0.009137967908589539, P67=0.006708801953668558, P68=0.003344392131161597, P69=0.0035463076491831893, P70=0.00669816499224521, P71=0.012679728678475074, P72=0.019902719216013786, P73=0.022810562585389018, P74=0.030557045777183944, P75=0.026542317273405884, P76=0.015754425053649827, P77=0.03626138676832976, P78=0.04537772169661668, P79=0.02649977575505013, P80=0.016752527874375915, P81=0.015892713349101653, P82=0.011634145146033654, P83=0.007849181164283556, P84=0.011589948162524075, P85=0.013718116586854701, P86=0.009086827425070216, P87=0.010562096013889492, P88=0.016360138537172525, P89=0.015668213827891086, P90=0.005871070885078768, P91=0.0018467319139459327, P92=0.008178215178122986, P93=0.017598724631517775, P94=0.030781339721628782, P95=0.015247597764735084, P96=0.000924013150126021, P97=0.002265488773175735, P98=0.0036634113431831344, P99=0.0031147536627051586, P100=0.002514607468722278, P101=0.0006863060345595436, P102=0.00017353115213313, P103=3.5814179996028986e-05, P104=0.00017326543272309008, P105=0.003015484212515179, P106=0.006095323562617645, P107=0.0035714163320615293, P108=0.0004533179502086751, P109=0.0004683908413320029, P110=0.001272741958989514, P111=0.0007823402635424886, P112=0.0003925775181038768, P113=0.00018108695389413694, P114=0.00223673950247411, P115=0.013106441650719515, P116=0.010823538956953467, P117=0.0011005522725199219, P118=0.0002590588923409405, P119=0.00023876096877167667, P120=0.001130687973780396, P121=0.0012349619193574187, P122=0.00043814353278056895, P123=0.0005138716277336841, P124=0.000419041734747909, P125=0.00022290275080230103, P126=0.0005413672972283663, P127=0.002769905264559719, P128=0.0009942186880502505, P129=0.00015536220867088022, P130=0.00010072391640974616, P131=0.00015956836256908, P132=0.0009777308411517676, P133=0.00036892919730853083, P134=7.09983900365007e-05, P135=0.00021864756002588513, P136=0.0004694865080297697, P137=0.0005212952541481196, P138=0.000462154347420388, P139=0.0005329302403649938, P140=0.0007612870468964283, P141=0.00040157807627120337, P142=0.00025741122968141617, P143=7.371491116796343e-05, P144=4.5833067114496794e-05, P145=6.226964773662063e-05, P146=5.5318723268003595e-05, P147=6.30880324968747e-05, P148=7.232419487264936e-05, P149=6.696818596431557e-05, P150=9.204083306404054e-05, P151=0.00015985417033623485, P152=8.397297773260899e-05, P153=7.054739932443e-05, P154=0.00011937990125764658, P155=9.208714970626973e-05, P156=0.00010959928502940963, P157=0.00010368734740610552, P158=7.39840787127796e-05, P159=8.057755919188092e-05, P160=7.403736408083796e-05, P161=5.571789216216776e-05, P162=5.3510096498401366e-05, P163=5.0882338809478694e-05, P164=4.294494733795395e-05, P165=5.4242380019299205e-05, P166=8.020358045314864e-05, P167=0.0001380581340181326, P168=0.0001151845884510043, P169=0.00011420334563312091, P170=0.00010073294183375053, P171=0.00016309194117489483, P172=0.00016593450157058402, P173=0.00016857467122216658, P174=0.00016148190346752746, P175=0.00021532003240834174, P176=0.00023132926994519544, P177=0.00015217245209258107, P178=9.087638452939872e-05, P179=1.4588154765885293e-05),Row( P0=-522.2157037829039, P1=12.07942201107562, P2=-52.90917254347038, P3=-13.16610292088927, P4=-32.138843958742896, P5=-17.713948192867925, P6=-5.802997286631245, P7=-3.7287873622108663, P8=11.077203827011902, P9=2.9342034835238224, P10=5.513415661698277, P11=3.9075355077049156, P12=-7.246543137125901, P13=3.7875133358835873, P14=-4.059759998430598, P15=2.9672098542932566, P16=-5.327413277603352, P17=5.059039357069013, P18=0.7490436448388443, P19=0.3255716476825476, P20=2.157209082174568, P21=1.7874189156775036, P22=-0.2643762445445259, P23=-0.5470829231652773, P24=-0.49540617920377666, P25=0.5895233484013896, P26=2.2384403959970967, P27=0.1503854661918173, P28=-1.4520361875207997, P29=2.5827316606188204, P30=-2.0445418286212385, P31=0.5440626585576199, P32=0.46713593980176826, P33=0.06319173785696727, P34=-1.8599011857364538, P35=1.5724995193161437, P36=-2.86232419764255, P37=1.8698292991490293, P38=-1.1034765389837093, P39=1.4694652148978806, P40=0.5605372984601629, P41=0.5487464685828628, P42=0.5292657525897037, P43=0.49648332244032645, P44=0.524617910530769, P45=0.4613164637389113, P46=0.4094581311502028, P47=0.4429390843747314, P48=0.48577429865019484, P49=0.5528184722488056, P50=0.5850060231543573, P51=0.5583751159534329, P52=0.0001077839260244279, P53=8.181403775525087e-05, P54=6.826431400506333e-05, P55=4.116395130731804e-05, P56=2.5516941834819788e-05, P57=1.2778351946757738e-05, P58=1.2116525320169016e-05, P59=1.5774918832319016e-05, P60=2.5300566700674195e-05, P61=4.144858257830807e-05, P62=9.279434864889088e-05, P63=0.00015265778466950152, P64=0.0004509639940887633, P65=0.001374777723699629, P66=0.001191630967757021, P67=0.0010569438374638566, P68=0.001109535566790723, P69=0.00256193630892089, P70=0.0035517294575419644, P71=0.004597810220137757, P72=0.008347256935763233, P73=0.008644722562070002, P74=0.0163673601506323, P75=0.024968606862096378, P76=0.03624050589985607, P77=0.0788059808993486, P78=0.12998707005881516, P79=0.19281806053267334, P80=0.3381609285318405, P81=0.2236772696534117, P82=0.09953210481544719, P83=0.08361161000227683, P84=0.1287903323767742, P85=0.12600979709599452, P86=0.07677999871462138, P87=0.10683939724340234, P88=0.14591814443757725, P89=0.20882590121182384, P90=0.27707572526074675, P91=0.3718545855901879, P92=0.1395734124349601, P93=0.03636383968182128, P94=0.056297071037751306, P95=0.07222625815936484, P96=0.04144612673431721, P97=0.07434657855511752, P98=0.07071552294182258, P99=0.0721099637351415, P100=0.1839739687353622, P101=0.36686143723900566, P102=0.09410654765368147, P103=0.03395759293597053, P104=0.06836979460762597, P105=0.17398216859107835, P106=0.14774671875201417, P107=0.11247214382611948, P108=0.15430199346351622, P109=0.18623406381710875, P110=0.10164687862621645, P111=0.16909943316540668, P112=0.21071902365040743, P113=0.11626217263662814, P114=0.07506427776557215, P115=0.01742545488205339, P116=0.02060019890448913, P117=0.0309696329320725, P118=0.03318780358138854, P119=0.02112265266273254, P120=0.014931080786572619, P121=0.015319404959439047, P122=0.04486158528973715, P123=0.06864397058592143, P124=0.04084240634405714, P125=0.020166650456667053, P126=0.007029748218120431, P127=0.03291471094610939, P128=0.0059049707073613004, P129=0.007859710704865986, P130=0.010823510536685373, P131=0.006201532803584265, P132=0.004898338359201141, P133=0.018109740886015805, P134=0.022165062271171012, P135=0.004207084615693283, P136=0.0015283087524152678, P137=0.0013299017094652236, P138=0.00194352293609098, P139=0.01297340600008416, P140=0.014346993536042761, P141=0.0029049154410475363, P142=0.0030329648816113035, P143=0.0024637292541984357, P144=0.002303462783858886, P145=0.0014703786445077471, P146=0.0018283688987883499, P147=0.0030347972486734517, P148=0.002230331936198319, P149=0.001351155204607033, P150=0.0018860163104422473, P151=0.002140224268809986, P152=0.0011758974444999622, P153=0.0014368256168329162, P154=0.0015733878251044712, P155=0.0010051392189992714, P156=0.0008004897272975625, P157=0.0002499836340993207, P158=0.0003081801860062155, P159=0.0002640349098595618, P160=0.00023808361866502697, P161=0.00029322939249223516, P162=0.00020383248404097452, P163=0.0002819900822255059, P164=0.00028444065832902834, P165=0.00031992342959484126, P166=0.00029623491106148074, P167=0.00024086014794591197, P168=0.00026982199002644207, P169=0.0002854765703042623, P170=0.00024952938629926587, P171=0.00029700855510834316, P172=0.0004141821174109691, P173=0.00038688013342857384, P174=0.0003626188483207103, P175=0.0002543696745764225, P176=0.0004540553323968012, P177=0.00035341557487971685, P178=0.00015934953928173898, P179=1.803581218009241e-05),Row( P0=-829.0647751058902, P1=40.807165038019605, P2=-7.88324589707452, P3=-3.022604239011615, P4=-16.81802754505774, P5=-10.53925073078122, P6=-7.301800398321502, P7=-3.1848374858516255, P8=-4.8064477523867275, P9=-10.333817490492036, P10=-5.2846696245874965, P11=-1.5806934769360181, P12=-4.998163652377846, P13=-3.5080222437719764, P14=-1.3163510195375172, P15=-0.38704801220479707, P16=-0.7570185667386166, P17=6.222814237028915, P18=4.876186454814538, P19=8.52432425948454, P20=7.649541745740755, P21=6.624907585322652, P22=2.222096037659629, P23=-3.799912613632693, P24=-1.2294474613437925, P25=-2.465362311901497, P26=-0.8419236307621416, P27=0.6387296225374057, P28=-4.232280054087508, P29=-1.4860196694448413, P30=-2.602906111943586, P31=-0.7734920394548295, P32=-0.36458036785059766, P33=-1.7949787119595846, P34=-2.466493188117973, P35=-1.0545151135884017, P36=1.0468387121423428, P37=3.675781170296644, P38=2.1564621733399276, P39=2.768203821294578, P40=0.5337696316455257, P41=0.5154312191187371, P42=0.4859975623362161, P43=0.4857549605103808, P44=0.5160002399830915, P45=0.5318061135211434, P46=0.5034672453381788, P47=0.5086000137928197, P48=0.5493585282108847, P49=0.6186937831010119, P50=0.6700890069243535, P51=0.5857589042954557, P52=3.0628995482639215e-07, P53=1.0142472634952834e-06, P54=5.991018986977042e-07, P55=2.0159994337240599e-07, P56=1.8784915457627661e-06, P57=1.1751514179612006e-05, P58=1.7802989506652997e-05, P59=9.276549277311567e-06, P60=8.619556311782402e-06, P61=1.2385390154970767e-05, P62=4.9290945744926265e-05, P63=0.0003997188515145767, P64=0.001515921436253389, P65=0.004006757331705746, P66=0.0022922717171705606, P67=0.00040261524528427325, P68=9.484247851744778e-05, P69=4.103083953191814e-05, P70=3.512371808538151e-05, P71=8.030869628481385e-05, P72=0.00011733908020455537, P73=8.094376854743373e-05, P74=5.985344458011755e-05, P75=7.620301434283821e-05, P76=0.00012521083007423808, P77=0.0003959440964548395, P78=0.0007780600502979415, P79=0.0011585215424557097, P80=0.0009921785904447348, P81=0.000248784852620903, P82=6.495428668491778e-05, P83=4.204907827433785e-05, P84=0.00011323754933507164, P85=0.00021555469569978483, P86=0.00014652362881416753, P87=0.00015662346278709938, P88=0.00016881680098554307, P89=0.00016750430067029217, P90=0.0002062348354369469, P91=0.00039835568271786095, P92=0.0006465136134597367, P93=0.00031019127602591217, P94=9.157726678749446e-05, P95=4.8137154870851376e-05, P96=2.688452523159355e-05, P97=4.511633443998501e-05, P98=3.300373537965271e-05, P99=1.81927199297792e-05, P100=4.553754764543692e-05, P101=0.0003477864709939686, P102=0.00013822710486836778, P103=3.5530641858774096e-06, P104=1.1194175776547187e-05, P105=3.7089487878756883e-06, P106=3.665658773584412e-06, P107=5.727591868517026e-06, P108=1.5065177382208764e-05, P109=5.00940673719984e-06, P110=4.080658735181097e-06, P111=5.920597007600297e-06, P112=9.681814508513287e-06, P113=7.800652625862358e-06, P114=1.0240320852534042e-05, P115=8.907751670010738e-06, P116=4.916506616519365e-06, P117=6.302308585855146e-06, P118=1.562576287479581e-05, P119=1.1922491831520411e-05, P120=7.037201415927525e-06, P121=1.1620835775725419e-05, P122=1.3192502603544633e-05, P123=1.37259888991865e-05, P124=8.923600718163493e-06, P125=9.044108424294291e-06, P126=8.194706902965019e-06, P127=7.2871604843126006e-06, P128=8.856350163158666e-06, P129=5.8998089030098455e-06, P130=2.4737806504448433e-06, P131=1.5576143659350755e-06, P132=1.249563291032889e-06, P133=9.108826310080757e-07, P134=9.522909492932296e-07, P135=2.0412504989472798e-06, P136=1.429197198672503e-06, P137=2.635955591492226e-06, P138=3.820928567960398e-06, P139=5.440699762783586e-06, P140=5.167593916301471e-06, P141=3.330571303049719e-06, P142=1.7784030261559019e-06, P143=4.78664211984141e-06, P144=3.948987138198385e-06, P145=8.647033374636376e-06, P146=9.341342087374076e-06, P147=5.457802173244392e-06, P148=5.883055532568637e-06, P149=4.007988351323198e-06, P150=3.4522286686822914e-06, P151=2.8185431737989685e-06, P152=3.3317531243068117e-06, P153=3.459382507557244e-06, P154=3.363711071361685e-06, P155=5.6162973937070185e-06, P156=7.575450855267603e-06, P157=9.667691671531049e-06, P158=9.476376737530651e-06, P159=5.631272329734491e-06, P160=3.2933189790957937e-06, P161=1.517629181626643e-06, P162=3.988188091160383e-06, P163=4.168659299288072e-06, P164=5.0183405490661e-06, P165=9.109156704341152e-06, P166=9.014624432648885e-06, P167=8.424055413360899e-06, P168=5.466682223870972e-06, P169=5.2301405366119404e-06, P170=5.550535231644218e-06, P171=3.4867818219487466e-06, P172=3.2706857563055547e-06, P173=3.892865388787335e-06, P174=2.7340685917385758e-06, P175=3.233594030662702e-06, P176=4.096462034005347e-06, P177=4.898570216885872e-06, P178=3.5181190982431297e-06, P179=2.7252844277996343e-07),Row( P0=-595.4663269763438, P1=25.50298739101821, P2=-27.33466435487401, P3=-8.492022048485868, P4=-24.49821540059539, P5=-15.72442489821913, P6=-9.689103167072773, P7=-10.886664842702196, P8=-2.853675091052965, P9=-8.95420006117585, P10=-4.840062110897457, P11=1.6823415062428757, P12=-1.0230975310502388, P13=6.475618105504393, P14=-0.9221616338736078, P15=5.466076759487472, P16=-1.9632341397325594, P17=1.0107468938091742, P18=0.45880834472908577, P19=-2.012434750795043, P20=0.07035835016982205, P21=-0.4698115082947295, P22=-1.2459946424718464, P23=-0.07179003717958164, P24=1.6335504125507385, P25=-1.6256010750478738, P26=-1.4478252959653644, P27=-0.48543723242765086, P28=-1.5395749078153913, P29=-1.1008457426077476, P30=0.06895581229657967, P31=2.0064416575329296, P32=1.2587662110374007, P33=-0.31059651104949504, P34=-2.6524745387300412, P35=0.8055832923706927, P36=-0.09572952023766056, P37=0.691442365386958, P38=-1.0579689946108985, P39=-1.6290724495216649, P40=0.480533150869161, P41=0.4874868374290044, P42=0.5059338491741133, P43=0.48265712445977393, P44=0.4932572492562918, P45=0.5011223385466893, P46=0.49935071804300984, P47=0.4567979111628666, P48=0.4155849699294951, P49=0.4445144804249712, P50=0.45555751773847886, P51=0.4726930061583754, P52=8.18808322929549e-07, P53=2.284289497335984e-06, P54=1.6833692986556906e-06, P55=2.186128896794588e-06, P56=9.291037914011823e-06, P57=3.5527364140310665e-05, P58=4.5729125525888934e-05, P59=4.228326383964689e-05, P60=9.067421595436869e-05, P61=0.00039745003290228743, P62=0.0016516402806044325, P63=0.0029043492079154788, P64=0.004360390801843235, P65=0.019360871985428428, P66=0.05287188948832544, P67=0.03744872876504588, P68=0.03365792916518096, P69=0.07388465828949488, P70=0.06769750691453262, P71=0.06912612232173933, P72=0.12499083867497152, P73=0.15616315399298644, P74=0.18276662486809747, P75=0.21796761387955405, P76=0.0951172554658918, P77=0.02581862804707167, P78=0.011379600433077829, P79=0.012026292284540652, P80=0.02152016496545722, P81=0.06523459483063158, P82=0.12415104656252925, P83=0.23350258544954636, P84=0.3820837483340176, P85=0.18638253452821865, P86=0.07325283433812434, P87=0.05388890271469885, P88=0.0436287662304357, P89=0.04129292258185302, P90=0.05517842002758574, P91=0.07137801931377186, P92=0.04090722949206074, P93=0.02185226352861795, P94=0.04054589709705548, P95=0.021585498136984613, P96=0.006318125059665932, P97=0.003965120000534676, P98=0.0287393250588359, P99=0.19242201485601465, P100=0.23440574912365925, P101=0.13150061139460786, P102=0.051852544491760126, P103=0.014730110767042566, P104=0.005701912974767457, P105=0.006010435327231687, P106=0.007528362390578669, P107=0.0077181076239014774, P108=0.021686853796530015, P109=0.04175709117826818, P110=0.02542589995567679, P111=0.008375026397154841, P112=0.006741417798730309, P113=0.007090581274086558, P114=0.025591728744417563, P115=0.056983283621630874, P116=0.02220904624222016, P117=0.011928790180296791, P118=0.0049636035894079725, P119=0.0025244798213786097, P120=0.003606930675494775, P121=0.007647545504777479, P122=0.0018273933311716989, P123=0.000651949605540057, P124=0.003033402544221485, P125=0.009714959953537628, P126=0.02980692232345912, P127=0.03482911010038082, P128=0.0019106426155142777, P129=0.0038915900479075407, P130=0.0029399906085478293, P131=0.0003216013504372614, P132=0.000748102405661721, P133=0.0007971757306619395, P134=0.000730095495745581, P135=0.0008509070383514658, P136=0.0016448853590888088, P137=0.0011257695278231179, P138=0.003215631572815658, P139=0.0044911613603718864, P140=0.002125829380905801, P141=0.0015276824574672885, P142=0.0008569622282889387, P143=0.0007274135807641007, P144=0.0008555879835998839, P145=0.000887211546989632, P146=0.0012038416137478646, P147=0.0010472952292993434, P148=0.0020078038629801134, P149=0.0013954458720034713, P150=0.0010560125412205216, P151=0.0008929253077500209, P152=0.0007018358723206417, P153=0.0006823863345937571, P154=0.0008757995758745063, P155=0.0006061078558051851, P156=0.0004740481381017918, P157=0.00017684898922901892, P158=0.0001514745915673756, P159=0.00013318482189815346, P160=6.0154747047713736e-05, P161=2.7724680156559906e-05, P162=3.0436477241672746e-05, P163=4.526067043974491e-05, P164=5.675480348298182e-05, P165=7.605469096647879e-05, P166=5.405496643474633e-05, P167=7.131511988715587e-05, P168=6.895683779994259e-05, P169=7.211645386856464e-05, P170=7.911081123868962e-05, P171=6.64140678765671e-05, P172=2.8887139781719795e-05, P173=1.925399382881269e-05, P174=1.9421141635752093e-05, P175=2.6549457579970174e-05, P176=2.5836193735709958e-05, P177=2.1751981331804448e-05, P178=1.6597676908845142e-05, P179=2.134819129886935e-06),Row( P0=-817.3491162209363, P1=31.045592304446583, P2=-15.360125411371454, P3=-2.401593119564055, P4=-21.57718643020242, P5=-12.36505613281632, P6=-7.533571353545324, P7=-5.562691990357036, P8=-6.32442905623165, P9=-10.288866180011297, P10=-5.993143762132988, P11=-5.281500184032532, P12=-4.195696262143463, P13=-5.934610328841807, P14=-0.6072632500249869, P15=-0.9977204999858279, P16=1.3396848362981773, P17=6.883019215996246, P18=7.000575105548152, P19=9.660119966324782, P20=6.522049862281002, P21=6.093227500620211, P22=1.1936803291127929, P23=-2.318543889615456, P24=-1.9251381894332, P25=-3.025321566875568, P26=-2.1291581823715164, P27=0.105731834970078, P28=-3.008782522539238, P29=-0.9162996107422638, P30=-2.2649949579915667, P31=-1.12862386077363, P32=-0.9070104589329707, P33=-0.7648522697867755, P34=-1.5300452482754006, P35=-1.310958833376777, P36=-1.834768977760195, P37=2.775709316462364, P38=1.791977971691619, P39=3.2767509116510487, P40=0.5073435114641069, P41=0.49693741267992264, P42=0.4526480741349929, P43=0.44424176208324545, P44=0.4828051290206736, P45=0.46116574593258886, P46=0.4622169952287944, P47=0.47275652905753945, P48=0.5172759801312651, P49=0.6028729327651781, P50=0.5791455581549199, P51=0.5281491328172764, P52=1.2711631709754344e-07, P53=2.432942682454777e-07, P54=1.6549113458303614e-07, P55=3.838591445361721e-08, P56=4.933214365264021e-07, P57=3.425597193208813e-06, P58=2.4856922133800232e-06, P59=5.765976044934153e-07, P60=1.0894625144360116e-06, P61=4.276051307767137e-06, P62=9.776381162224399e-05, P63=0.0006186455076261219, P64=0.0023469277781724726, P65=0.004405658370104346, P66=0.0036111343747534438, P67=0.0023226126049344345, P68=0.0012951084792742929, P69=0.00018523252015048282, P70=5.560069024795725e-05, P71=1.8193592937546196e-05, P72=5.951548605934839e-06, P73=1.3391874224146575e-05, P74=9.03763178562376e-05, P75=0.0001315038149911603, P76=0.00023162460541948138, P77=0.0015379202543342157, P78=0.003754307976393075, P79=0.002478921284558509, P80=0.0009426932476590367, P81=0.0006561725834206295, P82=0.000500300703927262, P83=0.0005127504821733363, P84=0.001448458998001924, P85=0.0011148636942563339, P86=0.0001414452128332848, P87=5.7857839954929556e-05, P88=0.00017402049633922272, P89=0.0006050609079233628, P90=0.0018979634730993502, P91=0.0018079388921328405, P92=0.0003317138121859546, P93=0.000288226756885392, P94=0.00020872054989850902, P95=0.000206376605060432, P96=5.3404338378059865e-05, P97=5.710642944186447e-05, P98=0.00012478980460705352, P99=0.00029198878701522856, P100=9.06981324450837e-05, P101=0.0003440013369009973, P102=0.0003240685299224425, P103=1.1441619949074438e-05, P104=4.029311988660528e-06, P105=1.9358470436297664e-05, P106=4.6933616836649405e-05, P107=1.6273760947877516e-05, P108=5.9168198097264687e-05, P109=2.603396772298838e-05, P110=8.088387698617162e-06, P111=1.4477807327612936e-05, P112=1.4312821695879203e-05, P113=1.1278711866889269e-05, P114=2.1522772047699495e-05, P115=1.122559771782167e-05, P116=1.04902970523422e-05, P117=1.004517862940284e-05, P118=2.32301255643978e-05, P119=2.6171760371960004e-05, P120=1.7928823093929716e-05, P121=1.7160426442392037e-05, P122=1.0799438264813305e-05, P123=1.769331595304511e-05, P124=1.937772654439785e-05, P125=1.1809266169119856e-05, P126=1.0358556332279237e-05, P127=2.6448085173940178e-05, P128=7.231379530660308e-06, P129=6.932260645636857e-06, P130=1.9756128473413313e-05, P131=1.3989566246869644e-05, P132=3.6336502352303617e-06, P133=3.7436589159588074e-06, P134=3.984917343843624e-06, P135=5.217927322640801e-06, P136=4.9678970357689245e-06, P137=7.654431149880194e-06, P138=5.903202811371621e-06, P139=1.2919722785064072e-05, P140=1.050780727842405e-05, P141=7.72917864102379e-06, P142=7.0535861839519186e-06, P143=7.981677516035738e-06, P144=1.241945854741432e-05, P145=6.160845316810033e-06, P146=6.2150188744750545e-06, P147=5.335028527350134e-06, P148=9.161087491676233e-06, P149=9.406177818365134e-06, P150=1.23882491017692e-05, P151=4.273662397500825e-06, P152=2.004113448234672e-06, P153=2.8823524278637153e-06, P154=4.424022756737716e-06, P155=2.8036943069092257e-06, P156=3.8852376152856825e-06, P157=5.4709527027004255e-06, P158=6.750933392752588e-06, P159=4.866641194230774e-06, P160=4.15708807576755e-06, P161=3.9801338788850475e-06, P162=2.5352136185816964e-06, P163=2.0648689123711802e-06, P164=2.238100515466986e-06, P165=1.6469928248255313e-06, P166=1.310214301817131e-06, P167=1.7943960226661326e-06, P168=4.1791831877668315e-06, P169=4.40346942769038e-06, P170=2.9924153029950394e-06, P171=1.264277300926093e-06, P172=1.5161062950682402e-06, P173=2.9592727415700764e-06, P174=1.7258181861434213e-06, P175=1.6431995520606076e-06, P176=1.9246946730540073e-06, P177=1.7330835002808326e-06, P178=1.1705464696911706e-06, P179=1.7464816949679774e-07),Row( P0=-706.6293563254321, P1=52.4286059278399, P2=-21.30827023625011, P3=-6.704208063688963, P4=-14.270531120374777, P5=-8.27319537524577, P6=-1.965703313106202, P7=-1.2059108610404703, P8=-7.392886145878012, P9=-9.29961052363673, P10=-4.326928819237173, P11=-4.152033812688998, P12=-3.7319063512674564, P13=1.3874273915739608, P14=3.4185144076732983, P15=3.481045143964762, P16=-0.25849456657076586, P17=2.801034879271406, P18=2.816603644483998, P19=2.9333523854417143, P20=2.3566882388547703, P21=4.523055924942122, P22=0.23093356231053505, P23=-0.8936203077768561, P24=-2.4663683663719502, P25=-3.2039172042075172, P26=-4.362939174011262, P27=0.43847653535975545, P28=-0.39369421183498515, P29=0.9051232046379952, P30=-1.6047180885319867, P31=0.2308542584643595, P32=-0.9901207374878774, P33=-0.1293066948005066, P34=-1.8782385486834263, P35=-1.8593990904771382, P36=-4.1232112059405805, P37=-0.9274732349275027, P38=-1.0789063620191899, P39=0.7835282323161237, P40=0.6134111911633537, P41=0.6063109666161987, P42=0.5997092779838743, P43=0.5846485374504707, P44=0.5943484396178575, P45=0.5525176981224784, P46=0.5411749180335191, P47=0.5442762395558914, P48=0.5758739197285132, P49=0.5486034222746861, P50=0.5420526961750911, P51=0.5584648272416509, P52=4.04936535663948e-06, P53=1.2932776811392161e-05, P54=3.2667468506472615e-05, P55=3.5078903293336626e-05, P56=3.691386636895972e-05, P57=1.782491110268859e-05, P58=5.784660762533549e-06, P59=6.92398312371014e-06, P60=1.3946131139035922e-05, P61=5.595814245466051e-05, P62=0.0003392071569495198, P63=0.0009440889942227686, P64=0.0017192164452360206, P65=0.0029311060261920036, P66=0.007076070895667863, P67=0.010540509873015011, P68=0.007921012494983785, P69=0.00610854421054367, P70=0.003223788771763097, P71=0.0007794072477817586, P72=0.0013256854080515904, P73=0.0013554295159889074, P74=0.0004792627406497882, P75=0.0004898736552268192, P76=0.0021155532195999043, P77=0.007510872357968389, P78=0.005013805498210904, P79=0.002241299239184676, P80=0.0016049172149697791, P81=0.0017142788176333942, P82=0.0016560597553861001, P83=0.0017814933146550853, P84=0.002510721095347155, P85=0.00437930641735896, P86=0.0033898228802696794, P87=0.005301437960062125, P88=0.007844260126435234, P89=0.008922662810073647, P90=0.005934134319502235, P91=0.004123178496085939, P92=0.004899758646500793, P93=0.0016835684207393946, P94=0.001909874204550304, P95=0.0018095286143840008, P96=0.0008577302215034329, P97=0.0018478472183439368, P98=0.002126540522884297, P99=0.0009326445982308392, P100=0.0009285223031338128, P101=0.00103092063584503, P102=0.0007734404929966977, P103=0.00039220372718123627, P104=0.00039215168324264934, P105=0.0004083752053163776, P106=0.0005985088369804264, P107=0.00025930256740254757, P108=0.0003314639173489295, P109=0.00017474537898656044, P110=0.00021806328884684857, P111=0.0003532391587944644, P112=0.00044937705343448523, P113=0.00010585119053013759, P114=5.800329986114801e-05, P115=0.000228249707628637, P116=9.140291688560415e-05, P117=5.241716815673552e-05, P118=4.253737732135668e-05, P119=0.00013082911520719256, P120=9.864187888973008e-05, P121=4.0882301328485824e-05, P122=6.935798270836083e-05, P123=0.00021297845103260783, P124=0.00012776109512687092, P125=0.00018581105498000888, P126=0.00028957345564884147, P127=0.00011963397685111687, P128=7.702868339529623e-05, P129=7.936191792321841e-05, P130=8.353297336432925e-05, P131=5.599447384132555e-05, P132=3.0160654133099174e-05, P133=3.5736483543643566e-05, P134=6.028577512946226e-05, P135=3.961323727601656e-05, P136=4.2602447941226486e-05, P137=3.5922401465335516e-05, P138=3.667140257050721e-05, P139=3.2295037204523e-05, P140=4.174915580747802e-05, P141=4.14140089896168e-05, P142=3.4267577808458336e-05, P143=1.4088353389685951e-05, P144=1.4391694144949253e-05, P145=1.7480062830296455e-05, P146=1.93697676776144e-05, P147=4.3924990958196145e-05, P148=2.8862416916976012e-05, P149=2.2011574955952e-05, P150=2.2011146327682294e-05, P151=1.4042650033314331e-05, P152=8.617965452549424e-06, P153=1.7434583322446345e-05, P154=5.517844854548167e-05, P155=6.100281026539648e-05, P156=7.309612653454595e-05, P157=4.686868695705401e-05, P158=3.8721628700044266e-05, P159=2.1795644013992888e-05, P160=1.986617400102995e-05, P161=1.149768584926103e-05, P162=1.3796743714227458e-05, P163=1.589222280830156e-05, P164=2.360429100232788e-05, P165=2.814534081145041e-05, P166=2.515207366753535e-05, P167=1.6489994111284144e-05, P168=1.8858554595439327e-05, P169=1.671968055892115e-05, P170=8.024074394916303e-06, P171=5.165181772523976e-06, P172=5.399323122485668e-06, P173=3.764734593840878e-06, P174=3.2928812158277917e-06, P175=3.778364911671281e-06, P176=4.291575567213476e-06, P177=4.803616188103469e-06, P178=2.222588604726253e-06, P179=3.4517723358004546e-07),Row( P0=-842.7009775786672, P1=39.22259362017119, P2=-10.74297775794878, P3=2.410892745832056, P4=-19.166386708346785, P5=-11.776287692177396, P6=-8.06561934491568, P7=-6.7178395776691, P8=-4.849973569259212, P9=-10.4187047587048, P10=-7.495030208843121, P11=-5.246388459997571, P12=-6.546057406775824, P13=-5.224285364818138, P14=-0.7285786117856333, P15=0.1924846770012553, P16=-0.350603490047774, P17=9.484626275084617, P18=7.267226461092107, P19=9.809305249164987, P20=6.893995012899968, P21=6.651052127825679, P22=1.5217921722328454, P23=-2.6899918579262203, P24=-1.8062833735091097, P25=-3.838553573633244, P26=-2.58994048689607, P27=-0.5084601306714714, P28=-4.355705697890909, P29=-1.1245641520531837, P30=-3.5482661195588903, P31=-2.375799129011305, P32=-1.4501852442757632, P33=-2.125768414985601, P34=-1.0382488846182425, P35=-0.6204621624271002, P36=-0.8355320427798852, P37=2.590546698006881, P38=0.5770392134169481, P39=3.3162858916895686, P40=0.4563778643746772, P41=0.42786329415381386, P42=0.40992367903396176, P43=0.41290152587817397, P44=0.4102939758075296, P45=0.40860825536323997, P46=0.4453424059355166, P47=0.5001074554668157, P48=0.5683910969166768, P49=0.5667438737998934, P50=0.5441911285247323, P51=0.5133405763259458, P52=2.3870974445561624e-07, P53=2.5603566229492395e-07, P54=1.4678136401235508e-07, P55=5.173993514949796e-08, P56=4.041081697870045e-06, P57=3.480055842544028e-05, P58=1.5908967978895932e-05, P59=6.810428874804947e-07, P60=4.784937668488787e-07, P61=1.0994500618194438e-06, P62=7.650765949345752e-05, P63=0.0006988760367742441, P64=0.0013017016647598758, P65=0.0024391064481360672, P66=0.003423710135732488, P67=0.0013671102630193018, P68=0.00010761369118212143, P69=4.36621863430676e-05, P70=5.2859982944742226e-05, P71=1.868851789260507e-05, P72=8.68728468582322e-06, P73=7.89131802924386e-06, P74=1.5595910530892023e-05, P75=5.0585566880378614e-05, P76=0.00016308557902840776, P77=0.0005129497298932143, P78=0.000441031225108117, P79=0.00020731932497240624, P80=0.00024505836861294506, P81=0.0006983666034942118, P82=0.0005503678128043663, P83=5.57530896232774e-05, P84=2.1047833780746936e-05, P85=2.4185258757676954e-05, P86=1.367741390754297e-05, P87=5.7026601410424776e-05, P88=0.00017654347281840398, P89=0.0002066633173044393, P90=0.00014208523234722625, P91=5.484897470195948e-05, P92=1.8968853628162712e-05, P93=1.6203990731323417e-05, P94=3.553325513393501e-05, P95=7.4735922000204164e-06, P96=3.078990426231002e-06, P97=1.3671724916561219e-05, P98=6.043117509037591e-05, P99=4.5434951792230634e-05, P100=1.985183780547746e-05, P101=5.989020947449658e-06, P102=2.259653638052395e-06, P103=7.277548874758462e-07, P104=2.1912469493991844e-06, P105=1.2792775411245977e-05, P106=2.0522252531703232e-05, P107=1.0699748246114034e-05, P108=2.2486499297454293e-06, P109=2.0914545897206613e-06, P110=4.5322626737497955e-06, P111=1.2205470475861771e-05, P112=9.565128321470306e-06, P113=2.8495164052019616e-06, P114=1.4998017764170872e-06, P115=5.317091261977912e-06, P116=4.413436490997878e-06, P117=7.374371703625077e-06, P118=4.70100906384968e-06, P119=6.1716539688637e-06, P120=8.264117161949597e-06, P121=6.1438895780561115e-06, P122=3.831030183652265e-06, P123=8.542232362964977e-06, P124=1.0123559518207634e-05, P125=8.042569057401199e-06, P126=6.8186269663311755e-06, P127=9.466634169563407e-06, P128=8.423027794158385e-06, P129=2.964691014754515e-06, P130=2.849638669353168e-06, P131=3.2200355363103578e-06, P132=1.0871356286756653e-06, P133=1.4711916194417141e-06, P134=1.0675499098715711e-06, P135=8.572448116935007e-07, P136=1.4737374800167166e-06, P137=1.2508048735778109e-06, P138=2.1530826620514507e-06, P139=4.303046346720346e-06, P140=3.811543635449881e-06, P141=4.98074393425879e-06, P142=3.038194752424489e-06, P143=2.845149230042851e-06, P144=1.6522274007279792e-06, P145=2.12446966665055e-06, P146=3.740679601719656e-06, P147=3.4095345309845744e-06, P148=3.7364464429581967e-06, P149=2.565505493899733e-06, P150=2.3911332988967785e-06, P151=1.282854120938513e-06, P152=6.13358372407474e-07, P153=8.067611057617666e-07, P154=1.4012287272882078e-06, P155=2.109735761574405e-06, P156=1.1165320445365704e-06, P157=1.150952424537792e-06, P158=1.5411250643168395e-06, P159=1.4735568206911387e-06, P160=8.697838413560591e-07, P161=4.990829036898628e-07, P162=5.915555952270004e-07, P163=6.87073431498399e-07, P164=9.131468704380746e-07, P165=5.920018813139795e-07, P166=8.993647372348026e-07, P167=8.514945880166166e-07, P168=8.006903573050178e-07, P169=1.5611362367554923e-06, P170=1.4642818785054737e-06, P171=1.5495081878329712e-06, P172=2.1328174015337325e-06, P173=2.844271764498114e-06, P174=1.6808587653768305e-06, P175=1.4752547069794199e-06, P176=1.2991342451582865e-06, P177=1.6073656140790905e-06, P178=1.0635151114879095e-06, P179=1.4863517069561224e-07),Row( P0=-409.7557003327414, P1=-4.076289731600233, P2=-53.16505164092557, P3=-20.068172997584103, P4=-14.618319490084405, P5=-7.0566940956235245, P6=4.162704508567999, P7=1.2550156196280784, P8=2.9916881230355137, P9=-3.205092044664918, P10=-0.43405439695550796, P11=0.32436600525249587, P12=-3.5116198534529195, P13=-7.673660921112906, P14=-5.551521746027082, P15=5.071705656155724, P16=-0.14275077044763748, P17=3.855497158008922, P18=0.5675502253371346, P19=0.9216228997159972, P20=-0.26602555284217133, P21=-3.6384924625369965, P22=0.23981923880350642, P23=0.807370727847684, P24=0.6302577866326362, P25=-1.4720133690343715, P26=-2.141663134023601, P27=3.1724426936890637, P28=-1.2563646260537933, P29=1.3417085355482894, P30=-0.9687734924781991, P31=1.3757332396124147, P32=0.25167684290082104, P33=0.9759070841379422, P34=-0.2114575652970647, P35=1.7820392929499655, P36=-3.920339172082215, P37=0.9267093760989854, P38=1.2659554280661183, P39=-0.18863338318439696, P40=0.6698127916516853, P41=0.6091084323191873, P42=0.5760468854656013, P43=0.5902862221000109, P44=0.5768771312471774, P45=0.5549689713534576, P46=0.6068055401212362, P47=0.6668817335925091, P48=0.6935650945840147, P49=0.6406783725554707, P50=0.6364706614234557, P51=0.7000218878237359, P52=7.698262555506531e-05, P53=3.0320896084098695e-05, P54=2.1118643014393352e-05, P55=4.241277462468609e-05, P56=0.0001558559566432527, P57=0.0008893753376867023, P58=0.0016828346177475528, P59=0.0010146576342010924, P60=0.0007610245716189721, P61=0.00116929410673267, P62=0.00117205937424041, P63=0.0009572852702946815, P64=0.001560090473888231, P65=0.0036044503472448734, P66=0.004412105609774234, P67=0.002987277641304475, P68=0.0016669797883749208, P69=0.0016314294422661547, P70=0.0016264264760295706, P71=0.0013422718808913429, P72=0.0013523260922175856, P73=0.0015111929031381669, P74=0.003095713515888986, P75=0.005388099113348456, P76=0.010909753189297869, P77=0.04637867439477979, P78=0.14802269596976064, P79=0.3267090684907424, P80=0.46506966481605744, P81=0.5851403873501545, P82=0.7762190642212087, P83=0.6885693335494881, P84=0.810521082484072, P85=0.6629949647816032, P86=0.6070034992442706, P87=0.5299063894455595, P88=0.5920873657332285, P89=0.8635818045771612, P90=0.713801893161301, P91=1.0559558522174572, P92=1.81381540116134, P93=0.8856665212789139, P94=1.8438830938516135, P95=4.559476419005291, P96=13.251753543358856, P97=19.14809969887113, P98=11.922494753627912, P99=14.122079903167341, P100=16.42007574457712, P101=6.006614308916063, P102=3.567728562369167, P103=6.977722312141868, P104=7.497484507614261, P105=2.917392672158868, P106=3.2233998950152003, P107=2.3160073447470886, P108=2.572850025200011, P109=2.3535352461422385, P110=4.926598943914502, P111=2.364925547025193, P112=2.3548384092956165, P113=3.753617454913539, P114=2.672254596170592, P115=2.628982425439018, P116=2.292745786740672, P117=2.624891367278723, P118=3.103361320526468, P119=4.414877301157507, P120=3.273427467078288, P121=1.1564559701663701, P122=0.7182771978764455, P123=0.14795629335026433, P124=0.1312065325070597, P125=0.1406220955993023, P126=0.24450095574406377, P127=0.31507866723582706, P128=0.08963951683598545, P129=0.08577520221114152, P130=0.07925385704955894, P131=0.05554270951246416, P132=0.1465251808851398, P133=0.15314939126622404, P134=0.09884934873240414, P135=0.06289573819939294, P136=0.05788435972304343, P137=0.06288730165517736, P138=0.11914999014937984, P139=0.3286599728541069, P140=0.3244686160480811, P141=0.21776926356288354, P142=0.17546716933679848, P143=0.17263566649844433, P144=0.03379918095855722, P145=0.049792437122371296, P146=0.057810644882300845, P147=0.025976194901792083, P148=0.021104193310923395, P149=0.023931546928262364, P150=0.019588535719097945, P151=0.007540418502243289, P152=0.005910133793679367, P153=0.013891571128332161, P154=0.023916035123624092, P155=0.026615528471964803, P156=0.0151050904116118, P157=0.006405548626767625, P158=0.002830368156401329, P159=0.0022568369300223342, P160=0.0025180709822420783, P161=0.002742575763123025, P162=0.0013472304161123048, P163=0.0008329046122504615, P164=0.001554578567048024, P165=0.002445479618253079, P166=0.0028608064656997366, P167=0.0041388330331084385, P168=0.004416283409466863, P169=0.004477667195376248, P170=0.004156655096259155, P171=0.0034629944159251455, P172=0.0030431904609841643, P173=0.004088122446805412, P174=0.004735737227496945, P175=0.00554007193445954, P176=0.005531394138229971, P177=0.005390368358817621, P178=0.0025615313397958574, P179=0.00029719344356332893),Row( P0=-510.86267695682824, P1=7.521859099264535, P2=-42.110000322091025, P3=-13.70382603331633, P4=-27.683531667193332, P5=-11.791033210849166, P6=-3.596452770517785, P7=1.8343632101652767, P8=3.8060921742590583, P9=-1.746435014475107, P10=2.580170963319787, P11=5.747281277519552, P12=2.7002466944475305, P13=1.9751188676501565, P14=-1.0642610998595037, P15=1.273434942374944, P16=-2.536830976604246, P17=5.429708682081591, P18=1.1582627447964582, P19=-0.37474477041298493, P20=1.6557473460676526, P21=2.404716329205346, P22=2.1343838153902444, P23=-1.589605150609913, P24=0.5072555711835501, P25=-1.0478299661887156, P26=-1.131744146513245, P27=0.43974306723257806, P28=-1.0647979552873434, P29=1.0403296749661712, P30=2.370600900211579, P31=1.4358460211340995, P32=1.0163767068062308, P33=0.5606462113114796, P34=-0.9085290599892996, P35=0.701264254031703, P36=-2.000383151195552, P37=1.7302634475798897, P38=-0.4602697567034438, P39=1.0626068588813375, P40=0.5048362763704685, P41=0.49880107941797125, P42=0.4895276062136486, P43=0.46117998220994305, P44=0.4600960778325221, P45=0.48783031282905676, P46=0.5046801113015349, P47=0.4976254499019274, P48=0.5123558716155643, P49=0.5099632603447601, P50=0.46335075771792883, P51=0.4826244204860739, P52=0.00010742199372038186, P53=0.00011504593221580639, P54=3.401049266165961e-05, P55=5.589149113682479e-06, P56=1.2017175984027308e-05, P57=1.4870568357353053e-05, P58=1.2163715390629735e-05, P59=2.2648081740866033e-05, P60=9.700652996009607e-05, P61=0.0001243087494354764, P62=0.00014566092240622083, P63=0.0002607826975953479, P64=0.00036631283791527117, P65=0.0011938146277365527, P66=0.008607041303092498, P67=0.023959243070645113, P68=0.0196288899683315, P69=0.020342402398103785, P70=0.015499509930646117, P71=0.013717155717800529, P72=0.03457680071698694, P73=0.04557704891213808, P74=0.10128513345812765, P75=0.10553284524109648, P76=0.13868729875131114, P77=0.3362145119707066, P78=0.4160012624296436, P79=0.3796174583291938, P80=0.3032057005806853, P81=0.42178062129733385, P82=0.5023743448749826, P83=0.4019530924421449, P84=0.3068418368776716, P85=0.28626137945554003, P86=0.2917702324993876, P87=0.428258647369895, P88=0.6239900375709405, P89=0.5894075570566758, P90=0.5587048546809926, P91=0.7586729653742724, P92=0.67327069097181, P93=0.45860388796440316, P94=0.5686426468285704, P95=0.9196333657946915, P96=0.627140289033325, P97=1.2007767231888968, P98=2.932029467514636, P99=1.3737408475511694, P100=0.6991343987593972, P101=0.9259304424112554, P102=0.7083903540299807, P103=0.21929555980106194, P104=0.06047925644086247, P105=0.060179789388183726, P106=0.11531995813266231, P107=0.08688717633079629, P108=0.05243931810490696, P109=0.11873986652117574, P110=0.1656162889338596, P111=0.11795361085167366, P112=0.151873760184916, P113=0.09967689734968146, P114=0.1264705566254686, P115=0.09216171450914193, P116=0.03653117086784132, P117=0.020298172966613466, P118=0.07894336233066612, P119=0.41239243297186906, P120=0.22488263810004394, P121=0.018631687571023323, P122=0.01393420114007271, P123=0.0365130331502752, P124=0.10305968778492118, P125=0.15080539130795653, P126=0.06242554553443652, P127=0.05476586641839821, P128=0.05644770794548614, P129=0.04278040089557966, P130=0.02419528435425728, P131=0.027966655827086687, P132=0.018046570951379667, P133=0.00782041609054182, P134=0.01230845876006229, P135=0.011481467114946969, P136=0.01689082814235784, P137=0.014465633877504032, P138=0.022125481235418073, P139=0.02232711868522825, P140=0.024663657147619467, P141=0.025858683782239655, P142=0.020227378683663866, P143=0.007073603897033197, P144=0.00569469135079285, P145=0.005377369363612915, P146=0.007486313697858209, P147=0.013496545997247679, P148=0.008689454936639201, P149=0.0053293388580230696, P150=0.006116438819665472, P151=0.002926294133786407, P152=0.0020655682194991422, P153=0.0018507464526475296, P154=0.002103328527034845, P155=0.005062923789457908, P156=0.005130740501514265, P157=0.005638808757065702, P158=0.0031895892091171854, P159=0.0008068106744664554, P160=0.00037321541856525476, P161=0.0004638857706222175, P162=0.00035862576061841385, P163=0.00035273322162245475, P164=0.0002750251773513976, P165=0.00031105734903339973, P166=0.00032970486553717893, P167=0.0004141518419607582, P168=0.000369515796170909, P169=0.0003636725027983114, P170=0.0002611588172599003, P171=0.00023234730051281105, P172=0.00022458744792182623, P173=0.00023574499800417496, P174=0.0002644095971349809, P175=0.00021405530413549155, P176=0.00032036036813758184, P177=0.0004034227672470303, P178=0.0002420564571593665, P179=3.704225978931385e-05),Row( P0=-838.3031175584214, P1=35.545553707568835, P2=-5.949400652877602, P3=-4.77610835725155, P4=-15.203508853648056, P5=-12.401483017031326, P6=-11.407795009983104, P7=-5.245093957236389, P8=-5.471216696586505, P9=-8.796828596274278, P10=-7.552890138898759, P11=-3.0075587562298267, P12=-2.636432267076908, P13=-0.07242620699632793, P14=2.3475358845185705, P15=2.549292996429827, P16=-0.7480144432581075, P17=6.802194518433225, P18=2.6588052372601876, P19=2.980351321712588, P20=4.642675953586121, P21=6.20226636121165, P22=1.949676562792121, P23=-1.4345895048345232, P24=-3.481895859786058, P25=-5.53933531992293, P26=-3.0503114425719997, P27=0.8031129824024236, P28=-1.0773776804399482, P29=0.12276530606233275, P30=-1.8536481083781824, P31=-1.252316330794531, P32=-0.4822811364179156, P33=-0.9679558205403883, P34=-1.978923155555894, P35=-1.849300586231224, P36=-2.3808443529061294, P37=1.1144822058943802, P38=0.610382400507972, P39=3.773817864843625, P40=0.5218499538915, P41=0.505588102695866, P42=0.5242923671550008, P43=0.5087751816897828, P44=0.4729431342342089, P45=0.4481447867194117, P46=0.4767771083519926, P47=0.5557541994736239, P48=0.5778465316571813, P49=0.5232984771641833, P50=0.4841185157908734, P51=0.502794652582634, P52=2.625840023320969e-07, P53=5.683227061581072e-07, P54=2.8777670161275946e-07, P55=1.0232592337218354e-07, P56=1.216191698991152e-06, P57=1.3622077570935902e-05, P58=1.5495732037145714e-05, P59=3.120085952631873e-06, P60=5.981142736480367e-07, P61=4.55662577421369e-06, P62=9.373377604258473e-05, P63=0.0008729891820091081, P64=0.001699196432316884, P65=0.0014079658435482245, P66=0.0009843858182450563, P67=0.0008494118230882671, P68=0.0005706259851510157, P69=0.0011571867548087014, P70=0.0015439525124440016, P71=0.0005555025877558664, P72=9.769533248304883e-05, P73=3.671659076341779e-05, P74=4.060062114010537e-05, P75=9.68619279180625e-05, P76=0.0004019473602985924, P77=0.0009681521884643505, P78=0.000799153733066208, P79=0.0006206926224076421, P80=0.00022896562312546152, P81=9.263359266014872e-05, P82=9.15454611267151e-05, P83=0.00014030977087825698, P84=0.00025564278719658395, P85=0.0004280196447247111, P86=0.0002563712750499215, P87=0.00025666120311089024, P88=0.0006284588137880453, P89=0.0005880297918489484, P90=0.00028087686105239, P91=0.00026031803134787714, P92=0.00011768726573534596, P93=3.166632972443492e-05, P94=1.806562093203845e-05, P95=1.4944250034105111e-05, P96=9.90684378146264e-06, P97=0.00012314906447488452, P98=0.0001467626128266946, P99=0.00010395173158507257, P100=0.00014726699393851513, P101=0.0001409016724571377, P102=2.881657043426347e-05, P103=2.318034313735109e-06, P104=1.8293051877544367e-05, P105=1.001330733286911e-05, P106=8.76943855259359e-06, P107=9.452346247667951e-06, P108=1.4550934527608247e-05, P109=4.161868021097829e-05, P110=3.5528005129915484e-05, P111=1.2335489759934234e-05, P112=9.790841496450893e-06, P113=4.147290585432315e-06, P114=9.89157481169073e-06, P115=2.9510711134824062e-05, P116=3.0924410029314095e-05, P117=2.070152955126786e-05, P118=9.01788488590574e-06, P119=1.1378661275784893e-05, P120=1.4842062026353085e-05, P121=1.2084925737666688e-05, P122=1.6763470705018896e-05, P123=1.4970512619570336e-05, P124=6.077651531632217e-06, P125=8.832156793259289e-06, P126=1.0002100377130963e-05, P127=1.0090404875058408e-05, P128=8.112849329605714e-06, P129=2.7722747601864354e-06, P130=1.359977089909009e-06, P131=9.674186397146855e-07, P132=1.1374216547616576e-06, P133=1.0298539478410038e-06, P134=7.748647868527804e-07, P135=1.4403882174927154e-06, P136=1.6217647949507706e-06, P137=3.5066198372127042e-06, P138=4.943609713809841e-06, P139=4.313884888645939e-06, P140=7.860000876674075e-06, P141=4.456661622100043e-06, P142=3.5335495190581563e-06, P143=8.789272970558465e-06, P144=4.255327485202954e-06, P145=2.0196763961779733e-06, P146=1.971822189811229e-06, P147=1.9702478950539793e-06, P148=3.75243802032338e-06, P149=4.610138826317726e-06, P150=3.6247299917843384e-06, P151=1.9669881124274887e-06, P152=2.9826708173200787e-06, P153=3.881147501990828e-06, P154=9.319627358132066e-06, P155=1.0600015744722858e-05, P156=1.5638803360800984e-05, P157=1.193614836096817e-05, P158=9.725928846403994e-06, P159=1.0152660946470501e-05, P160=6.515164403362861e-06, P161=5.382686614955627e-06, P162=4.177423419016734e-06, P163=3.3586912684034107e-06, P164=3.7653650237688343e-06, P165=3.5504953851623824e-06, P166=4.54216803922258e-06, P167=5.24611918117506e-06, P168=3.837009552230142e-06, P169=4.789441733881648e-06, P170=5.853770266276226e-06, P171=7.611945518315358e-06, P172=6.5462466633656816e-06, P173=4.672142331188664e-06, P174=4.168165565355013e-06, P175=2.803348348799931e-06, P176=3.7910565304251094e-06, P177=4.631990489610619e-06, P178=2.4154910199416488e-06, P179=4.6368381115272887e-07),Row( P0=-575.3654342117377, P1=57.88497908370352, P2=-14.462899191034774, P3=8.510282198013282, P4=-11.165703374294985, P5=-14.861135175190254, P6=1.8825501466730208, P7=-9.328318020686272, P8=-1.4237469898769317, P9=-4.204793665487529, P10=-11.193033592593677, P11=-2.2914656109685807, P12=-2.403747443107328, P13=-7.927825273377565, P14=-3.401323259249134, P15=-2.373329503144863, P16=-5.496941646677627, P17=-1.1636338476801902, P18=-3.303979739506491, P19=-1.059536681403405, P20=-6.623182434744127, P21=-2.310919328739446, P22=-4.840813268226178, P23=-2.489199529571306, P24=-3.8632213891609424, P25=-2.5927857760851025, P26=-5.172051611343544, P27=0.7464179354912844, P28=-2.1854157657038438, P29=-0.36237721700228887, P30=-1.1702727379460895, P31=-1.5541749532372338, P32=-3.093988328396088, P33=-0.6792568326722992, P34=-1.2714041879548026, P35=0.5283625026312805, P36=1.1148977867272243, P37=2.4612987038179153, P38=1.7578467716875452, P39=1.4231583754245882, P40=0.6686873060109264, P41=0.6663858798955545, P42=0.653071306434376, P43=0.6378563940218954, P44=0.6687289426766619, P45=0.7725735288418755, P46=0.7935756222268807, P47=0.8040146556945765, P48=0.7979662092982255, P49=0.8172653491636336, P50=0.7662860203751888, P51=0.6679604433977525, P52=5.305546256825328e-06, P53=1.7358483153707322e-05, P54=4.958139792896997e-05, P55=0.000244529814894774, P56=0.004583689255373299, P57=0.024034365125186228, P58=0.020327879900021956, P59=0.02498470391601436, P60=0.04254462646988243, P61=0.042450976990300655, P62=0.032830610117832455, P63=0.02914235679525244, P64=0.01743373144332667, P65=0.04226726033060061, P66=0.051080285249948926, P67=0.04246950559872685, P68=0.06525268886528618, P69=0.06378245117995039, P70=0.067166676726492, P71=0.06254447755749915, P72=0.04080605806853977, P73=0.03965324590014508, P74=0.023459663466289138, P75=0.027091041464660372, P76=0.08648418397590629, P77=0.1473481934461986, P78=0.13956601418334846, P79=0.14707701834296483, P80=0.15396929401781706, P81=0.11339324848082731, P82=0.05632575195451921, P83=0.018694885959908102, P84=0.025473302822402854, P85=0.041167530517511076, P86=0.07474701128670648, P87=0.0723612392753226, P88=0.03416018626612321, P89=0.01598201840565909, P90=0.011395672241190674, P91=0.03469968394338532, P92=0.1468582435347265, P93=0.09106914867254097, P94=0.08110485719923172, P95=0.0999777913425223, P96=0.11910405263948419, P97=0.12869257442894466, P98=0.1058206783704886, P99=0.11659694188330949, P100=0.041497591099799364, P101=0.029201653123980573, P102=0.00987791197691809, P103=0.0064156125048965736, P104=0.015483454433492744, P105=0.018152779322361992, P106=0.013359258714990153, P107=0.03125043990733936, P108=0.0377966693333014, P109=0.029124619366007843, P110=0.03593421204911696, P111=0.04167636471925575, P112=0.023459922656271327, P113=0.01850347193593623, P114=0.01648464141329564, P115=0.00722939533575195, P116=0.011076878014241022, P117=0.022640822339845327, P118=0.021191060862763244, P119=0.007979346889631601, P120=0.021479786955011074, P121=0.018761513423855233, P122=0.011540178952276665, P123=0.017917421496170188, P124=0.006236190222937021, P125=0.005937025104244434, P126=0.004773454864865279, P127=0.005462006716268341, P128=0.003272913006079896, P129=0.0031476704099422967, P130=0.0027620677571935977, P131=0.002576513539927387, P132=0.004588998418826686, P133=0.011715996945048253, P134=0.008295770518414897, P135=0.01459688826352623, P136=0.01676887133333239, P137=0.02026380138092845, P138=0.01655760762241812, P139=0.007790695172402473, P140=0.004263127765917748, P141=0.003661651507670258, P142=0.002731350574249117, P143=0.0026749988781594645, P144=0.0019789396134980443, P145=0.0017205377893945168, P146=0.0015368036378055868, P147=0.001432308714529895, P148=0.0003760189557256506, P149=0.00033645806982666066, P150=0.0003072906111719858, P151=0.000439783083986499, P152=0.0004835580515755125, P153=0.0005983362538528541, P154=0.0006007422876866914, P155=0.0008175334462884979, P156=0.0006834946394308788, P157=0.000566194324837753, P158=0.0004724478663424706, P159=0.0005623489287221176, P160=0.001132263141082897, P161=0.00045175101131354685, P162=0.0005397240961861501, P163=0.0004333144102784668, P164=0.00017104458460384328, P165=0.00019303521922506734, P166=0.00013529170861547972, P167=0.0001288564985641663, P168=0.00018351303008511122, P169=0.00016921728147757393, P170=0.00018571821771867563, P171=0.00011565779130948295, P172=0.0001762996496517355, P173=0.00012243169915271716, P174=0.00011363511079871103, P175=0.00012407066861896496, P176=0.00012141606869286725, P177=0.000157055338777573, P178=7.482234447485845e-05, P179=8.989248591394656e-06),Row( P0=-569.8242878987801, P1=59.13511194841781, P2=-11.56428674117427, P3=5.71840299454841, P4=-13.666203946805453, P5=-15.239841620536042, P6=-2.1281706970308676, P7=-11.641225725067931, P8=-3.6414713472013878, P9=-4.825805219683895, P10=-12.519985229843359, P11=-1.0938374164912243, P12=-3.793093528788741, P13=-8.974731208346672, P14=-3.886619449012951, P15=-2.215447829134949, P16=-3.981747650399859, P17=-1.9138574258832637, P18=-3.127787642500896, P19=-1.5921970974738422, P20=-5.961355898382046, P21=-1.738536686194251, P22=-3.6996431968423016, P23=-2.408764229706068, P24=-3.8478465940304023, P25=-0.8385877635625221, P26=-4.590790876222279, P27=0.687333421630224, P28=-1.0973987997638608, P29=-0.11121915433936141, P30=-0.5831039713062479, P31=-0.2453870649640342, P32=-2.243604057234174, P33=-0.04128251216216583, P34=0.33260912622498406, P35=0.2270576476783543, P36=0.16733930927774474, P37=0.4989106022511405, P38=-0.13279550475695426, P39=1.0236043544610451, P40=0.6758789928332549, P41=0.681304154529326, P42=0.6958301756469137, P43=0.6627274783228463, P44=0.6871560956545635, P45=0.7383815086031449, P46=0.7990090661989463, P47=0.791044318114132, P48=0.7851782334699511, P49=0.7531952194026865, P50=0.7148726261411308, P51=0.7078826754686408, P52=8.007816592012879e-06, P53=2.2759717675905558e-05, P54=5.7209221233069666e-05, P55=0.0002634454918825249, P56=0.006716402424608359, P57=0.03929049321393582, P58=0.02635742940276814, P59=0.019969081204999708, P60=0.04248240368050316, P61=0.03251408229861433, P62=0.027963662372276012, P63=0.049149507001259446, P64=0.02937089604477774, P65=0.03705556768427959, P66=0.049260186162794796, P67=0.04522514777282803, P68=0.08078119964389016, P69=0.22709521050381484, P70=0.1919341248433967, P71=0.06244720563644043, P72=0.021921822008332936, P73=0.023876437370727453, P74=0.025751303196315006, P75=0.042621726312052875, P76=0.07466949769058773, P77=0.2626669445188193, P78=0.46473743135973566, P79=0.26266917477463114, P80=0.07318995820469167, P81=0.041595934580937126, P82=0.06363012464712267, P83=0.058204728900535095, P84=0.03690940754889648, P85=0.02486694193304643, P86=0.029961482197446303, P87=0.043351055763620985, P88=0.03989180609153666, P89=0.020935035047541208, P90=0.012027419685410873, P91=0.021639876545910396, P92=0.07938013803660712, P93=0.052030257497317746, P94=0.034816388928192804, P95=0.047547683455452475, P96=0.1063679672704438, P97=0.0868045403674016, P98=0.057072803005959134, P99=0.053119351112609944, P100=0.0250486472078712, P101=0.026761760908209992, P102=0.011393924778417093, P103=0.008385677321635074, P104=0.010354822478529271, P105=0.0077766478140072325, P106=0.009877428687518881, P107=0.013751139050237431, P108=0.01106445770299401, P109=0.013919562975621962, P110=0.027143415125455046, P111=0.03460080127430966, P112=0.026581800332537472, P113=0.018207945331272242, P114=0.00684782358456104, P115=0.01709194444189785, P116=0.020380319280307542, P117=0.011087121347958973, P118=0.006894155007234042, P119=0.007967360660688884, P120=0.006637435865090164, P121=0.006628597166130317, P122=0.005113233680779363, P123=0.005675595702873424, P124=0.0026748984497288173, P125=0.002109525351023891, P126=0.0024479721802063447, P127=0.002595908263711224, P128=0.0026988174663032743, P129=0.0013061605336429668, P130=0.0009068744678686048, P131=0.0009007530141286795, P132=0.001209636314186185, P133=0.002175865608733139, P134=0.0030359042458418042, P135=0.013995716345426507, P136=0.015295682920565856, P137=0.01842373773091031, P138=0.011114082419418001, P139=0.0048841195984698615, P140=0.0038192222097781572, P141=0.004445929972047462, P142=0.005873185586746595, P143=0.0066708817534743324, P144=0.004319113611331787, P145=0.0023791731621107485, P146=0.0018147188750682317, P147=0.0011353561732032687, P148=0.0006458910816086837, P149=0.0007886604389968463, P150=0.0009003811355460262, P151=0.0009402506515083496, P152=0.00035006449615702326, P153=0.0007303413990474952, P154=0.0006564064969905952, P155=0.0003448901264039965, P156=0.0002231352340781342, P157=0.00046139935548426527, P158=0.0004176503464119711, P159=0.0004353236948360548, P160=0.00045206568089728807, P161=0.0004787076577191694, P162=0.00036363773110675824, P163=0.00032980059352457714, P164=0.00026367020747481584, P165=0.00022692396626441353, P166=0.00018836891122150378, P167=0.00017821390684681738, P168=0.0001533634627436987, P169=0.00013955752770117074, P170=0.00010695333225305351, P171=9.948641518898207e-05, P172=8.33682859558066e-05, P173=8.625231585356128e-05, P174=8.446336776045487e-05, P175=8.12760377486943e-05, P176=5.359950109922554e-05, P177=5.0564115667640504e-05, P178=3.5575517344154154e-05, P179=4.369941378689641e-06),Row( P0=-611.1002566799281, P1=71.59809643073407, P2=-7.458161460236205, P3=9.32321912921586, P4=-9.073475383912761, P5=-13.545230555731239, P6=1.7715420469796694, P7=-2.5790280893207065, P8=5.505077756019049, P9=-2.9721711431316256, P10=-8.264866558722384, P11=-1.6717223339449163, P12=-5.520948583745237, P13=-7.188919845351247, P14=-2.2343614573938857, P15=-3.124452153751609, P16=-5.197391226662749, P17=-0.04889149453099326, P18=-1.8034065986578107, P19=-3.022092121721349, P20=-6.392528265890786, P21=-2.2399796999677357, P22=-3.4298311097271506, P23=-3.8654413939845593, P24=-4.094379931097575, P25=-0.999087942346166, P26=-7.774608470190142, P27=-2.2549714678469814, P28=-2.1750403171002946, P29=-1.783364844439174, P30=-1.6131349959458468, P31=-1.07759562768364, P32=-3.15292791364185, P33=-1.2878907921362666, P34=-1.246415022173906, P35=0.2218183240981726, P36=-2.0317259659619387, P37=1.40011343427748, P38=3.4720176936970324, P39=3.5762492257315883, P40=0.7306619072583055, P41=0.6306332270519468, P42=0.5459289992222023, P43=0.5830742040703555, P44=0.654251500144772, P45=0.6629770680448726, P46=0.6703241037567672, P47=0.6696167684515385, P48=0.7205473715658453, P49=0.7643291421787217, P50=0.7334628677774322, P51=0.7138526034660237, P52=4.1285474013362745e-06, P53=1.4990852322514684e-05, P54=3.724021201515357e-05, P55=0.00034834280987487644, P56=0.005481247694473841, P57=0.03467193137702542, P58=0.04367335505411776, P59=0.02514547329106835, P60=0.006473629914323356, P61=0.0019751146114903484, P62=0.010411520289820729, P63=0.01234397488349555, P64=0.030990953042155663, P65=0.0320721108555661, P66=0.031499520474285884, P67=0.04489572664319936, P68=0.01992092072159448, P69=0.003208038289577688, P70=0.005855367090501833, P71=0.014021279021042847, P72=0.010839798620586382, P73=0.011424428634340063, P74=0.013290780126703433, P75=0.016232543437508126, P76=0.0177314380765623, P77=0.058763632597724096, P78=0.12416076584192634, P79=0.11049368195805798, P80=0.04488242090160554, P81=0.023246842312603398, P82=0.01656995378583396, P83=0.02753905246321003, P84=0.05228855099771623, P85=0.02970911112690243, P86=0.02607936452930196, P87=0.028256846218053685, P88=0.016299169185360708, P89=0.023347665021291663, P90=0.025565642445627694, P91=0.027721703409893876, P92=0.041416261981227755, P93=0.022026157454803998, P94=0.08733527874077444, P95=0.08952068571503936, P96=0.02727994422602264, P97=0.008067017362020487, P98=0.002690137176187237, P99=0.0008473856061124535, P100=0.002788032558921775, P101=0.0023617522308394447, P102=0.000738068581176126, P103=0.001788408486931299, P104=0.0009335901312879727, P105=0.0018445767247527308, P106=0.0019668621412815405, P107=0.0011118717129768745, P108=0.0013286347738230859, P109=0.001751969119971265, P110=0.005031693447486683, P111=0.014911713362752014, P112=0.008860047226417549, P113=0.0026900647707776395, P114=0.0009232839329124862, P115=0.0012259299012433496, P116=0.0008284492262432977, P117=0.001787927328097224, P118=0.0014769998951912203, P119=0.0012903423017172403, P120=0.00107476367039225, P121=0.0005527857441375992, P122=0.0005170712842399737, P123=0.00036993600676480824, P124=0.00016973307929021923, P125=0.00024620964632104643, P126=0.00019946616384066622, P127=0.00010729286561677252, P128=3.9931282795110437e-05, P129=4.599037386343344e-05, P130=5.2800651559985296e-05, P131=5.14423332167449e-05, P132=6.54695719172527e-05, P133=0.00019357528137051874, P134=0.00017967753125317496, P135=0.00024659739884936584, P136=0.00025007731298982175, P137=0.0001690449733963896, P138=0.00010353235002543489, P139=6.249523146611398e-05, P140=5.5218054881617004e-05, P141=0.00010414796327959922, P142=0.00011940828790720966, P143=0.00015828975687717654, P144=0.00016055264273737117, P145=0.0001542350383572019, P146=0.0001541440969687948, P147=0.00010328845962767417, P148=4.414903938418396e-05, P149=5.954611350440566e-05, P150=8.752791239103996e-05, P151=9.20635012989262e-05, P152=0.00011857787730880816, P153=0.00013198798597566434, P154=0.00022657506113442718, P155=0.0002906523273344905, P156=0.00029492813605804864, P157=0.00018230798627953535, P158=0.00015498895261221264, P159=0.00018889277621716552, P160=0.0003015217645003689, P161=0.0002750359192743653, P162=0.0001302529622720631, P163=8.770094423982819e-05, P164=6.849029918774141e-05, P165=7.513120639936439e-05, P166=7.160179368676752e-05, P167=8.201010473009716e-05, P168=7.565116763278737e-05, P169=5.041403958942583e-05, P170=2.936982443296587e-05, P171=2.473075090131781e-05, P172=2.441270658342715e-05, P173=1.983421783958062e-05, P174=1.7927298733932444e-05, P175=1.5795811743578073e-05, P176=1.8589959622193097e-05, P177=2.5596912507397216e-05, P178=1.4486825104687147e-05, P179=1.7876114385278314e-06),Row( P0=-624.7575614269633, P1=77.52918837174003, P2=-11.366561744373161, P3=9.778044953951722, P4=-11.29174206165166, P5=-14.281609135900068, P6=-2.4160381083274194, P7=0.4796721961549452, P8=4.1146385833180865, P9=-4.275228863909842, P10=-6.4557133596794065, P11=-2.3198242620234653, P12=-6.194256812713761, P13=-6.263660405206227, P14=-2.621410089714159, P15=-2.085884701014167, P16=-5.08574510738579, P17=-0.44938673800575507, P18=-1.876196840509254, P19=-2.582654639083709, P20=-6.404944495513933, P21=-3.184269828132956, P22=-4.207388202833929, P23=-4.1499164245779445, P24=-3.8026014404426545, P25=-0.09648164731521952, P26=-7.719095786059342, P27=-1.471457804355294, P28=-2.906832495582951, P29=-2.5983889013619654, P30=-2.447966182740837, P31=-3.139307224930139, P32=-3.870162177361022, P33=-3.016686213727817, P34=-0.9057397722560133, P35=0.09272139384958776, P36=-2.7809167749936465, P37=0.2604284620204512, P38=2.0313274071488916, P39=1.5110207637236497, P40=0.6946886852421135, P41=0.5981755803520383, P42=0.5981611331764595, P43=0.625089766675508, P44=0.637079261886252, P45=0.6456292457420986, P46=0.6876617532260763, P47=0.7235843657294473, P48=0.7402436080206265, P49=0.7165310539807058, P50=0.6703665115237425, P51=0.7089139765395427, P52=3.2638883057118825e-06, P53=1.0821049352842924e-05, P54=3.1222584892820544e-05, P55=0.00018018152478480702, P56=0.012293820345501706, P57=0.11694320342783113, P58=0.07565036737056617, P59=0.016749311262014566, P60=0.0029025369247010634, P61=0.0014311312505392863, P62=0.012471992385303027, P63=0.031105890177233433, P64=0.026381295480454578, P65=0.018472433263007355, P66=0.03110262476684352, P67=0.04429130269564351, P68=0.013573168973263649, P69=0.010060513853820006, P70=0.023551512355711606, P71=0.012092285729595847, P72=0.004811186861576545, P73=0.004850803632074535, P74=0.015465386853748984, P75=0.024893268153289915, P76=0.025661366645759066, P77=0.033825965132411664, P78=0.039856616986473206, P79=0.03712819287044833, P80=0.02336106966014864, P81=0.011029661910773807, P82=0.031751466828297555, P83=0.0353732785753406, P84=0.013463645371743826, P85=0.014192643112261713, P86=0.017963791693372515, P87=0.010415008597932776, P88=0.007893304989477637, P89=0.007209526351627472, P90=0.018412244557557928, P91=0.02204118014035517, P92=0.008235873911359154, P93=0.003589092595732992, P94=0.013712324647666316, P95=0.02489686630791482, P96=0.004909042399231068, P97=0.0008543559134761889, P98=0.0028800091479650775, P99=0.0025808532543902924, P100=0.00045706331202455795, P101=0.0006729067534589483, P102=0.0005976926448103188, P103=0.00024845748738217487, P104=0.0004309301124007673, P105=0.00025383052072334506, P106=0.0003130413843206174, P107=0.0006504367771720549, P108=0.00017139621456204493, P109=0.0006042500153916291, P110=0.000558386496722963, P111=0.001612627464634978, P112=0.002246802619608283, P113=0.0009137651208713059, P114=0.0008566305297157584, P115=0.0003386498157434959, P116=0.0004321838717361842, P117=0.0002984934578875401, P118=0.000373203950117188, P119=0.00026323055196634057, P120=0.0003896422209810411, P121=0.0003572229731155922, P122=0.00021904280352463915, P123=0.0004586427715448957, P124=0.00020981933577609695, P125=0.00016021852796923737, P126=0.00013685736074960369, P127=0.0001304523039789628, P128=3.425608053882744e-05, P129=3.381421500549271e-05, P130=4.5615681217410426e-05, P131=3.237663083754398e-05, P132=3.45772678191336e-05, P133=0.00011015548609004474, P134=7.746874594045171e-05, P135=0.0001369258334060539, P136=0.00010657108814419429, P137=8.256900172255548e-05, P138=5.1891663598441085e-05, P139=3.761023816774498e-05, P140=3.7543029301416344e-05, P141=4.680311597600204e-05, P142=6.944895583534526e-05, P143=0.00011451378450746618, P144=7.864302442457902e-05, P145=0.00015421511150120549, P146=0.00010750796828383914, P147=0.00020382305695721304, P148=0.00010184912253372405, P149=4.00509771804503e-05, P150=6.465359695867647e-05, P151=0.00014100618397716366, P152=0.0003394985078188561, P153=0.0003682764227482742, P154=0.0006602779806940545, P155=0.0006522397816411099, P156=0.0009503246177075977, P157=0.0009132993269239337, P158=0.0004421514331410425, P159=0.0003388556821812949, P160=0.0003109140057502852, P161=0.00032952653015761336, P162=0.00025722128825105754, P163=0.0001502776133393383, P164=0.00012195116472681, P165=0.00013354506249582053, P166=9.952205586435149e-05, P167=9.124996552804588e-05, P168=8.857152224822701e-05, P169=6.396652738167664e-05, P170=4.561405863184826e-05, P171=3.1302687983847856e-05, P172=2.6429580537779443e-05, P173=3.295924688861983e-05, P174=2.855907558803911e-05, P175=1.886815278694919e-05, P176=2.0680857630067307e-05, P177=2.693528750570361e-05, P178=1.510558899935919e-05, P179=1.8180745007452044e-06),Row( P0=-528.6186758904433, P1=49.587696945783534, P2=-15.8734323254176, P3=3.5040601746902746, P4=-13.416739009834009, P5=-19.15189840065949, P6=-7.579244247173266, P7=-9.462710413864315, P8=-6.461669155504644, P9=-3.7629533116882183, P10=-12.475806119384064, P11=-4.1905105656383865, P12=-9.276439859542982, P13=-9.129912626717822, P14=-2.4720212815292313, P15=-4.374105067212916, P16=-6.04590851511992, P17=-3.856171408352212, P18=-5.648494235204462, P19=-1.469606903722213, P20=-8.204469527005495, P21=-3.3302528465705317, P22=-1.8457219629849968, P23=0.9825449500148977, P24=1.5928626028024284, P25=5.0283610341268385, P26=3.6550215908543597, P27=3.711164112658572, P28=3.3150198496524355, P29=2.01300274034876, P30=2.1975662951944326, P31=2.117461917613475, P32=-0.4048345893059896, P33=1.955544297309892, P34=0.789159161164339, P35=0.8950342858679813, P36=-1.8395072746610837, P37=0.997366504193749, P38=-0.4028473616240392, P39=1.1645329832887712, P40=0.7501454839989056, P41=0.6929503948242758, P42=0.7083917270640808, P43=0.6842692710847996, P44=0.7215632320270654, P45=0.6476627878399237, P46=0.5897861546345566, P47=0.6076194077887512, P48=0.663063477168844, P49=0.6934314938232377, P50=0.7279608235177807, P51=0.7928641071295491, P52=1.1489708190387109e-05, P53=3.086685011304761e-05, P54=4.769497900358947e-05, P55=9.916318143308888e-05, P56=0.0003334621684495435, P57=0.002240245915039612, P58=0.011608170679227812, P59=0.07090549282130504, P60=0.2368569506916985, P61=0.6506969498315092, P62=0.7338261845460108, P63=0.21438955755648967, P64=0.06095913518967212, P65=0.02557651199367328, P66=0.03317800010356086, P67=0.08108488181647516, P68=0.17041571522562005, P69=0.41687167751410925, P70=0.4701310897275231, P71=0.4186406608370961, P72=0.7905242607511541, P73=0.5455490081163179, P74=0.1995785327550675, P75=0.22162784958681211, P76=0.36258551575682757, P77=0.6220842289024637, P78=0.8431964604940538, P79=0.6034077399022869, P80=0.2630757229498126, P81=0.312010423149565, P82=0.7633321475585296, P83=1.1763009167111769, P84=0.8127555477038523, P85=0.2863853346978881, P86=0.08071643524241337, P87=0.19755866482708037, P88=0.5634156038728281, P89=0.32990852883763255, P90=0.17549922025073036, P91=0.6223280598546896, P92=0.6458423002438736, P93=0.04634186516015805, P94=0.020006359642847738, P95=0.059812295568822324, P96=0.04007695845814202, P97=0.018069713880901204, P98=0.06807839798000302, P99=0.0845583235335542, P100=0.012569459282132071, P101=0.020587450491229942, P102=0.011776384978614862, P103=0.025707736617915693, P104=0.057833461808421555, P105=0.03718212244048689, P106=0.03410025722618409, P107=0.045326469740029095, P108=0.036401678016393185, P109=0.1127802659958075, P110=0.0807515898332592, P111=0.033059056528179434, P112=0.00910001584173986, P113=0.012960651604291612, P114=0.009068135266898106, P115=0.004293404541646608, P116=0.004343866551730116, P117=0.006716907608369484, P118=0.010379929656813242, P119=0.007465684594369222, P120=0.009739432084691333, P121=0.021944781697365568, P122=0.014371012591949893, P123=0.021883089812796297, P124=0.012188279226618198, P125=0.007195852015951115, P126=0.005534526562863868, P127=0.005157637524142577, P128=0.0030037381594760957, P129=0.0027519527255102315, P130=0.003060035007210769, P131=0.0015544886782335957, P132=0.00276577724678231, P133=0.004266164304658395, P134=0.006435670100547766, P135=0.019313501417634923, P136=0.014931147770476024, P137=0.007935005284342966, P138=0.005871363533794894, P139=0.00393243735533705, P140=0.0036811051998448595, P141=0.005653453839631745, P142=0.0070757152662189945, P143=0.009482432472018253, P144=0.006396401462872906, P145=0.0022547460617498944, P146=0.002048769153618054, P147=0.002954377081191158, P148=0.0013108157161550982, P149=0.0007971896626289947, P150=0.00048006902141173747, P151=0.0005459120012050302, P152=0.0008561446664142782, P153=0.0013857375011848958, P154=0.001704638239532775, P155=0.0022309852031570645, P156=0.0034817998238326035, P157=0.0025257635034079476, P158=0.0015203179475790168, P159=0.0018416861268512985, P160=0.00207484916666937, P161=0.002184604028257724, P162=0.002138292718103819, P163=0.0014530187638093844, P164=0.0014118873854357939, P165=0.001746409579438824, P166=0.000962442011633115, P167=0.0013387337667862657, P168=0.0012316681173215227, P169=0.0012041970422723561, P170=0.0010533528196629428, P171=0.0007982383342400001, P172=0.0007011072532621454, P173=0.0007888909946335947, P174=0.0008198710411166069, P175=0.0009028321807742899, P176=0.0006352511801533683, P177=0.0005888615422387884, P178=0.00035690056167950456, P179=3.3855527366809605e-05),Row( P0=-515.5284158710716, P1=45.79659612671133, P2=-24.072151912736654, P3=4.734064183710823, P4=-15.530060903983449, P5=-19.844273876966938, P6=-4.707767487623614, P7=-11.225487202375541, P8=-7.313047634171064, P9=-3.6456243182825583, P10=-16.574858117012564, P11=-3.376286306717114, P12=-8.024723210840854, P13=-9.13865684905292, P14=-1.8413857015562751, P15=-5.037797294672821, P16=-1.5690047982589908, P17=2.6136539047117773, P18=1.045158437972317, P19=1.790071849422913, P20=-2.6259519310081174, P21=-2.3497292359580277, P22=-0.43991730424709363, P23=-1.4238649228608906, P24=-0.6215049313550134, P25=0.4774896565361956, P26=-3.380532375653247, P27=-0.05590153736195589, P28=1.0394459299394054, P29=0.45425984950053594, P30=2.8683051741334418, P31=1.6279969988095053, P32=-0.6428245632997591, P33=2.2575226149851004, P34=1.063281064993109, P35=1.7363773878262063, P36=0.07129088978553769, P37=0.6235182607128533, P38=-0.6085133194366188, P39=1.2375075018225408, P40=0.7483688113087051, P41=0.6811640370778125, P42=0.6449093657020848, P43=0.6470074720318603, P44=0.6100522113959532, P45=0.6255279078631611, P46=0.674360112819518, P47=0.6633946078121009, P48=0.6449899558995515, P49=0.6821486537575785, P50=0.7167168540175337, P51=0.7439827997803602, P52=9.913076858191259e-06, P53=1.5425086520153586e-05, P54=2.245764302076787e-05, P55=0.00010031332999100251, P56=0.0006839296275511673, P57=0.005191043413752008, P58=0.016187942766658886, P59=0.037937649899583634, P60=0.05042114208903415, P61=0.06153577644195193, P62=0.15701244117134233, P63=0.21505660736473586, P64=0.132430488951713, P65=0.1627589289640086, P66=0.27881667634283785, P67=0.36294892797858525, P68=0.24514893461906612, P69=0.22214046653058392, P70=0.3117811284738459, P71=0.18061863764871566, P72=0.08398836919370402, P73=0.1254861114611655, P74=0.17197596551685357, P75=0.16792854293241172, P76=0.17977216321012335, P77=0.42076874727631663, P78=0.732380512036038, P79=0.7245297439764729, P80=0.3672097631774707, P81=0.22224482661750775, P82=0.35502365924614876, P83=0.6384676957346811, P84=1.750873494759788, P85=1.4616620660457729, P86=0.16810366072855154, P87=0.10598159914082098, P88=0.12660186147500288, P89=0.15584987520049523, P90=0.1952304186780923, P91=0.267938449964246, P92=0.43126402483873744, P93=0.4224611174889095, P94=0.4395811227548871, P95=0.3561431061580338, P96=0.2244746164852116, P97=0.1283811937344979, P98=0.07123649572629649, P99=0.07284190399451929, P100=0.04626721269506592, P101=0.04390650548291288, P102=0.024884831098610435, P103=0.04594899667203852, P104=0.034897693829106596, P105=0.03551153321517998, P106=0.0366737038492945, P107=0.053208553313891724, P108=0.038166788133737965, P109=0.0541035936342771, P110=0.12150510561056846, P111=0.14437930900709955, P112=0.15770412237694156, P113=0.06623575975156434, P114=0.042653300470905785, P115=0.033188178098698326, P116=0.14435065811217185, P117=0.05827641210137866, P118=0.02330403416138251, P119=0.015925813835273986, P120=0.024757566692637463, P121=0.04214814267400095, P122=0.03875892516647346, P123=0.03527728219221855, P124=0.014820980375843435, P125=0.007697867440939615, P126=0.009310213299935801, P127=0.023396441454361337, P128=0.023109381659185184, P129=0.022283232402851544, P130=0.008656996657227132, P131=0.004796836129835823, P132=0.004010748840895153, P133=0.009723820551749321, P134=0.010654679057200382, P135=0.01865330759627048, P136=0.02320749049995426, P137=0.020121799882862963, P138=0.015896050106297355, P139=0.012915839452543447, P140=0.012815705812539427, P141=0.014427705059480319, P142=0.013199942806039625, P143=0.018260615598168557, P144=0.023454085349356626, P145=0.01612315814232911, P146=0.007204883669237185, P147=0.005145661223176575, P148=0.0009988287236945565, P149=0.0001862424198854202, P150=0.00017933859789122185, P151=0.00035328550677802905, P152=0.00023032145642767996, P153=0.00044887744648015773, P154=0.0008345381693051998, P155=0.000539203336594702, P156=0.0009584373011904126, P157=0.0013619708774243204, P158=0.0006225459447512432, P159=0.0008782627378318042, P160=0.0010552889782961816, P161=0.0009603691747175415, P162=0.0008114870259742262, P163=0.0005322593332300992, P164=0.0006379098849168745, P165=0.00029425903863667015, P166=0.00029898510958696535, P167=0.00042352910135367623, P168=0.0003690401219241051, P169=0.00019177314279122167, P170=0.00017482196450848268, P171=0.00016613604477301288, P172=0.00015988464172565664, P173=0.0002235844551748023, P174=0.0001550975258929656, P175=0.00016567636561777644, P176=0.0001874727667539573, P177=0.00018277078757741272, P178=8.880348974070887e-05, P179=1.1877669438478594e-05),Row( P0=-518.3767212835169, P1=45.33080851970608, P2=-22.84350880436229, P3=4.056662665377164, P4=-16.728393260782042, P5=-18.037522888761544, P6=-5.782143062467385, P7=-5.532020588016066, P8=-5.504569321863139, P9=-1.6356376449018248, P10=-13.831935565769626, P11=-4.328397705447121, P12=-6.69098974468333, P13=-11.138218057906338, P14=-2.100072334196035, P15=-3.960658708931538, P16=-6.164070677641631, P17=-4.852527018616077, P18=-4.916316532657296, P19=0.6523772552183542, P20=-6.013504124378615, P21=1.190944714484829, P22=2.5821747788059013, P23=3.1585023825961938, P24=3.838111210802092, P25=2.553779006455423, P26=3.654512717219306, P27=2.7132521012727753, P28=3.9826493575597395, P29=2.620940654108348, P30=1.1133470789688458, P31=-0.04352297689826725, P32=-3.6432791226819607, P33=-1.640749597820453, P34=-1.0468888530169034, P35=1.6988246482486313, P36=-2.1495933514174217, P37=0.19576688076374765, P38=0.25441951788002415, P39=-1.4283961044553795, P40=0.6953025180315537, P41=0.6493381158023138, P42=0.6350325194379294, P43=0.6712982937885844, P44=0.6769590905589192, P45=0.6609663616441597, P46=0.6647741859282825, P47=0.7086125326550491, P48=0.691922840881427, P49=0.6740692410055689, P50=0.6764907861328806, P51=0.7044120348133546, P52=8.427244869864732e-06, P53=1.9584857983621264e-05, P54=5.4630100890149415e-05, P55=0.00011647693786644747, P56=0.0004260608096253046, P57=0.0027637602337779936, P58=0.00746384900601168, P59=0.021163332256187804, P60=0.11275185253853727, P61=0.42415455437067434, P62=0.7575102596813745, P63=0.8920498428061095, P64=0.2718864423119016, P65=0.06731966255314065, P66=0.03124002646762925, P67=0.029981394120214864, P68=0.026787524191652434, P69=0.08445539587190878, P70=0.32378226111217145, P71=0.32193468429326133, P72=0.22673476236623996, P73=0.30489564321479634, P74=0.5735691954226508, P75=0.7358895913034288, P76=0.6014857039486855, P77=0.7057057012115808, P78=0.4042664345606699, P79=0.2403762887661112, P80=0.20371503048608175, P81=0.23117207170095, P82=0.25971965033558664, P83=0.22001934590037983, P84=0.2892776172979714, P85=0.23651784587622476, P86=0.29777092179111875, P87=0.4052003660698772, P88=0.33352858896385856, P89=0.3562738223957143, P90=0.39505865218645625, P91=0.7473551610103725, P92=0.8462771337244027, P93=0.16413293828870976, P94=0.1287659990142712, P95=0.20251337328035957, P96=0.18952564400138805, P97=0.15493444974783083, P98=0.30297893401827336, P99=0.14108390912313482, P100=0.05403260689628288, P101=0.04132056442670033, P102=0.0281297865694309, P103=0.03737134566271113, P104=0.019128801990418574, P105=0.011249556979292176, P106=0.05101099138811324, P107=0.14520637686831814, P108=0.047188067405859103, P109=0.02494479031798631, P110=0.07104630394179964, P111=0.1586304326649501, P112=0.17199479457420658, P113=0.15368107199835368, P114=0.03123730411035203, P115=0.01222018680628641, P116=0.046895374089117974, P117=0.03307545741110098, P118=0.017529205039923793, P119=0.0338064792463318, P120=0.018729159907275854, P121=0.02302175667555558, P122=0.023643723411946864, P123=0.008449442156593554, P124=0.0036966398557635306, P125=0.009004076059629888, P126=0.004247013072787177, P127=0.0034496818622872976, P128=0.002786028229702616, P129=0.0037660741096406996, P130=0.002631943649863993, P131=0.003533373721485771, P132=0.006327162858233539, P133=0.02789898812764344, P134=0.02435888958170797, P135=0.03558977195349897, P136=0.020723231004715383, P137=0.019173052088371546, P138=0.01407856604115692, P139=0.022808155297819, P140=0.025724110700147438, P141=0.03312654561481254, P142=0.01935435859657883, P143=0.012376846992696406, P144=0.008029422090429334, P145=0.0083836282884395, P146=0.004075021346485583, P147=0.0017418262082839137, P148=0.0005108998376095941, P149=0.0004128673797245616, P150=0.0003223906093358348, P151=0.00028880225521990153, P152=0.0002620942836077678, P153=0.0002544321548443499, P154=0.00026561404029934953, P155=0.00022589668917802458, P156=0.0003151168081415205, P157=0.0002301398269783428, P158=0.0003625962552755583, P159=0.0002889952745376182, P160=0.00044809451339500566, P161=0.0005728463599048185, P162=0.00040333667487469006, P163=0.00020898488880537175, P164=0.00015777992401293462, P165=0.0001672656430685503, P166=0.00017457377242692372, P167=0.00015427952110629752, P168=0.00014144958959086572, P169=0.0001168441011491846, P170=0.0001361056574339733, P171=0.00016587618634854886, P172=0.00011065200847089466, P173=6.280047907952056e-05, P174=6.546729131727686e-05, P175=5.709347111230518e-05, P176=7.750615208779101e-05, P177=7.629573110986701e-05, P178=3.723914208800165e-05, P179=5.260640761040263e-06),Row( P0=-522.4379131163291, P1=55.772703473784425, P2=-22.280174117075713, P3=0.9936372009264635, P4=-16.80652793608486, P5=-19.072032540084663, P6=-5.026471895839384, P7=-7.499761505142453, P8=-5.631509040252263, P9=-5.042166571423729, P10=-13.836197024232584, P11=-1.7591894019209793, P12=-8.088193384102627, P13=-8.091202325510318, P14=-2.508033125222166, P15=-6.573840825993324, P16=-3.9601745267520934, P17=-3.9053666437029673, P18=-4.012890883134274, P19=-3.4546686645723685, P20=-7.1922153893791325, P21=-1.8047814923538006, P22=0.19089128536486064, P23=1.2510830895467415, P24=2.4118616659274688, P25=3.8245489429419006, P26=-0.6273827472989455, P27=0.8434903954350947, P28=3.3878090137788917, P29=1.4456511434027872, P30=3.175326517032512, P31=2.361889632149008, P32=-0.8971150976466349, P33=-0.4553649220534285, P34=-0.4673728894205395, P35=1.3181638771933215, P36=-0.00983719780611388, P37=1.042300893310669, P38=-0.9760010514011368, P39=-0.610370517615707, P40=0.7543023064330545, P41=0.6963278757292324, P42=0.6630184356707162, P43=0.6492279446117971, P44=0.6187204358419522, P45=0.641571198895241, P46=0.6665186433956095, P47=0.6776893575538829, P48=0.626224095545309, P49=0.6345703985745069, P50=0.6754926958813616, P51=0.7328983509595685, P52=5.290269211025343e-06, P53=1.7491526379805282e-05, P54=3.7693339685350194e-05, P55=8.419464841291567e-05, P56=0.0011645678380254217, P57=0.007627457972555737, P58=0.01304379239229219, P59=0.04262721908654182, P60=0.11135432909079225, P61=0.20644539037106832, P62=0.3583932448409923, P63=0.32763370829824584, P64=0.05111853976886091, P65=0.012131931420988125, P66=0.047507064426104736, P67=0.1030651122376538, P68=0.12593252166212182, P69=0.21135028704713732, P70=0.43041019525030966, P71=0.30974923241454044, P72=0.215272565500674, P73=0.39631680384781437, P74=0.5083197537597857, P75=0.48951781639530256, P76=0.14850407555763095, P77=0.115336167713043, P78=0.1854644842105608, P79=0.3083138397674774, P80=0.5003079342635965, P81=0.5780135315563155, P82=0.5756464494412862, P83=0.5526838764044841, P84=0.7960707903712438, P85=0.6128398203008133, P86=0.5082133859475592, P87=0.40399442554987597, P88=0.44151678965032776, P89=0.3936712190198861, P90=0.14128662363176628, P91=0.10008811040895792, P92=0.3528708496663621, P93=0.2526700828935872, P94=0.49167916306423765, P95=1.6699667989799312, P96=0.9664275487678599, P97=0.15739560807658687, P98=0.15923461911758247, P99=0.1557755873741413, P100=0.04683512489267498, P101=0.041025045866943624, P102=0.0339591165908984, P103=0.015775635810522756, P104=0.01856781277893894, P105=0.028028000188793162, P106=0.036000514391521514, P107=0.044347658503158104, P108=0.02168544319655163, P109=0.022208465179876628, P110=0.047111064460625074, P111=0.04340454532547931, P112=0.09198758049407622, P113=0.09142163012996914, P114=0.03632460608594544, P115=0.021245427644509797, P116=0.036190140129364765, P117=0.06546887916153286, P118=0.0331180307534693, P119=0.008745916330482753, P120=0.031064872024321653, P121=0.03894876997172728, P122=0.015464533653842498, P123=0.02201273445016588, P124=0.008487140526612742, P125=0.005841786241823064, P126=0.015381729497622088, P127=0.008200766310722706, P128=0.003808011056055797, P129=0.0027288691674106835, P130=0.0015113052833138207, P131=0.001619480187051141, P132=0.002010498247109393, P133=0.005296105051364633, P134=0.004150177203089973, P135=0.006281573256180934, P136=0.007919833436663607, P137=0.003741534461786999, P138=0.0036703460453806234, P139=0.003013437133188318, P140=0.004558984879878349, P141=0.007765446873174165, P142=0.005201418715210014, P143=0.004360265887917332, P144=0.0030486503321227503, P145=0.0027440261287460703, P146=0.0013046226263053054, P147=0.0007138754458957655, P148=0.00047936613512415026, P149=0.00021464539406281744, P150=0.0006579170314287326, P151=0.0007959621564093523, P152=0.00043682778143132144, P153=0.0008066007719061214, P154=0.0013998253307015611, P155=0.000938856747734308, P156=0.0009458544124392199, P157=0.0012440432970628069, P158=0.0014235123150199815, P159=0.0012247319317227733, P160=0.0014333153647234666, P161=0.0019225966512805895, P162=0.0019520973708493264, P163=0.0016297738574547012, P164=0.0013610066549828294, P165=0.0007101757103788126, P166=0.0005975935933025841, P167=0.0005758756622156761, P168=0.0006226411353598958, P169=0.0007497506725648929, P170=0.00046921775832394017, P171=0.0002634164090321507, P172=0.00016771678743084638, P173=0.00012075750989604707, P174=7.00646957691789e-05, P175=5.4929931773847426e-05, P176=7.658755535565647e-05, P177=0.0001072891451691156, P178=5.5860622554805244e-05, P179=6.702672163915131e-06),Row( P0=-630.2406161750428, P1=74.60954591483551, P2=-3.290399269265022, P3=11.89167583788067, P4=-8.234938619691597, P5=-7.722442429896395, P6=2.280541673290191, P7=1.2913789847170705, P8=4.750377549564022, P9=-3.022210945933285, P10=-5.147311630335044, P11=-1.3394005729744547, P12=-2.7243750448763677, P13=-6.224393204837218, P14=-1.6677205098126842, P15=-1.7015703735103291, P16=-4.220925095006905, P17=1.7944152206031891, P18=-1.6601185137124261, P19=-2.9814245780644537, P20=-6.281306277036555, P21=-2.847787207431114, P22=-1.418719929413345, P23=-2.6296100251889514, P24=-4.188898978313513, P25=-0.4421353806329815, P26=-5.237670350274375, P27=-0.3457238034291115, P28=-1.7690323846786993, P29=-0.5421192522689069, P30=-0.3191652821721662, P31=-1.272270364117667, P32=-2.2808690621688985, P33=-0.4757551043144685, P34=-1.1526860783036688, P35=1.0310500312685056, P36=-2.3365166351329276, P37=0.5169027782500772, P38=1.6266835565894922, P39=1.8097188237123334, P40=0.6901115267148588, P41=0.6171226857907611, P42=0.6133541927630788, P43=0.6568570216597611, P44=0.61460075309327, P45=0.6530798047191851, P46=0.6966040023438124, P47=0.7285464236724511, P48=0.7632602073353756, P49=0.7558317724305079, P50=0.7276706554490654, P51=0.7270674159328206, P52=1.820215231837035e-05, P53=6.465446395614635e-05, P54=4.643463362362054e-05, P55=0.0002276029122989854, P56=0.010791295075405298, P57=0.1034966708595535, P58=0.07115567454825561, P59=0.024221015682421857, P60=0.009576153706831113, P61=0.0027253399638941414, P62=0.006554970156995474, P63=0.012940378141789008, P64=0.011751104604952576, P65=0.013603174977022223, P66=0.03445440389997084, P67=0.03966588542776537, P68=0.023782600371630058, P69=0.011515140191205406, P70=0.020962707560457955, P71=0.008971818425110082, P72=0.004105686480999198, P73=0.02047103048210743, P74=0.023127074421613384, P75=0.022486354515743677, P76=0.02833707646758069, P77=0.01933552196423782, P78=0.01547871140959323, P79=0.013985184475362289, P80=0.025316962404027952, P81=0.04331176998098699, P82=0.028830391100938074, P83=0.024774751071032, P84=0.03575318404950207, P85=0.01621309496889257, P86=0.002441755921050006, P87=0.0029920536820818478, P88=0.011456435376627652, P89=0.005502444173433349, P90=0.002337557574634637, P91=0.0016752145013577838, P92=0.013404088047552883, P93=0.027497592448418897, P94=0.007709759775817911, P95=0.0030127809298003063, P96=0.004413525191297752, P97=0.00863800524382473, P98=0.0026589250984739654, P99=0.00044473208856746866, P100=0.00049673070928253, P101=0.0009769917156072674, P102=0.00016913766057872053, P103=0.00024672295610632324, P104=0.00044254646821791256, P105=0.0001670379647096825, P106=0.00062066197091727, P107=0.0011790752550058987, P108=0.000162056048839358, P109=0.0010090802028529856, P110=0.0010576400261449062, P111=0.0009413476512125724, P112=0.0015835892455064817, P113=0.00041839378774781596, P114=0.0004922673589712665, P115=0.000218487914699405, P116=0.0002474545477516182, P117=0.00022604731708900571, P118=0.00028626142233127553, P119=0.00015419095640895164, P120=0.00037322002139925645, P121=0.0003151987489004911, P122=0.0004494589057926776, P123=0.00047794074300070447, P124=0.000170227083023991, P125=0.00014960634047268516, P126=0.00014158234881673788, P127=0.00022769578022107927, P128=8.096400658354372e-05, P129=5.7080039662477345e-05, P130=3.930948099196248e-05, P131=3.315587907337372e-05, P132=2.207317466296294e-05, P133=4.250941605931028e-05, P134=4.335107821866228e-05, P135=0.00012478159418278592, P136=0.00016437357839738133, P137=0.00010180923242124467, P138=5.861193897990033e-05, P139=6.72342009913646e-05, P140=5.4137802330599494e-05, P141=0.00010254183584397118, P142=0.00010312214553869695, P143=8.089281039066649e-05, P144=8.26487677978412e-05, P145=7.481225358359709e-05, P146=8.943890502435425e-05, P147=7.688118313841163e-05, P148=3.78482797227568e-05, P149=1.9461816684832437e-05, P150=1.984809495807488e-05, P151=6.64195341989434e-05, P152=8.28103469117821e-05, P153=0.00016840131223214939, P154=0.00041828988114332526, P155=0.000396561090095886, P156=0.00019362819358212561, P157=8.454427920794756e-05, P158=8.005218406311613e-05, P159=6.899651598458527e-05, P160=0.00018879330695437322, P161=0.0002300633739606795, P162=0.00015331400327051243, P163=7.263414172309059e-05, P164=6.83117165541889e-05, P165=2.609497010604751e-05, P166=2.6519053875050656e-05, P167=5.3939424432342895e-05, P168=4.613552204302781e-05, P169=4.375331947079287e-05, P170=4.0710282114350044e-05, P171=2.3303351761986923e-05, P172=1.8383156384700156e-05, P173=2.0728257455881686e-05, P174=2.517058947044596e-05, P175=1.8573879396725863e-05, P176=1.490921101831906e-05, P177=1.7503313093527153e-05, P178=1.1190452109593369e-05, P179=1.2261496403943926e-06),Row( P0=-575.5354191615187, P1=58.48902880466526, P2=-12.604670553199824, P3=4.9570923529827615, P4=-13.615124662655619, P5=-19.006400403836437, P6=-2.1907430115207185, P7=-8.506110424137445, P8=-4.505344746827415, P9=-4.331998727834207, P10=-9.851694318675923, P11=0.16014983240325897, P12=-5.935962301770792, P13=-8.535934958471165, P14=-2.9238554804153774, P15=-1.8161355815449165, P16=-3.4812080168514554, P17=-1.42543266111608, P18=-3.609633884133235, P19=-2.0158167822781317, P20=-6.284790426179461, P21=-2.291086225352383, P22=-6.212475435276428, P23=-2.3577188868439243, P24=-4.550726228920487, P25=-2.090299561129781, P26=-3.9962801245800934, P27=1.408245614409494, P28=-0.17279957763274403, P29=2.499829374438051, P30=-0.6456108732649056, P31=-0.8148754489638141, P32=-0.2601872767170478, P33=0.48607163194064923, P34=0.8179331484162615, P35=1.1985116399238795, P36=0.7148153458469698, P37=0.26616407980630447, P38=0.11802958283675598, P39=1.947888475557958, P40=0.7044982602733425, P41=0.6604736347362081, P42=0.6751666834308819, P43=0.6844815499976795, P44=0.6408838691964187, P45=0.6371998898441525, P46=0.6918044964732564, P47=0.7615999798750189, P48=0.7423309108092241, P49=0.7683730451944959, P50=0.7768078466058052, P51=0.7383390680035332, P52=5.129997043578881e-06, P53=1.5672899088574406e-05, P54=2.8279968930179805e-05, P55=0.00010729062970217361, P56=0.004478895989683491, P57=0.033468807678938055, P58=0.026061925235520624, P59=0.01962625092903191, P60=0.024409868094521645, P61=0.03351747116648792, P62=0.028513010430361148, P63=0.03864211026335006, P64=0.024088791118953384, P65=0.03209060988860471, P66=0.04977424735237818, P67=0.060271607524005434, P68=0.049200214781430635, P69=0.07916394887247159, P70=0.1291884203290305, P71=0.0837186673199464, P72=0.03304545526241707, P73=0.03163240832243665, P74=0.022843681569300514, P75=0.034661861154424156, P76=0.06551784294206298, P77=0.12999746592736494, P78=0.189092363527576, P79=0.2440310740989167, P80=0.25544705942509777, P81=0.14815912202871945, P82=0.08560320926685468, P83=0.05103194265487606, P84=0.04569266964096543, P85=0.03300669196633752, P86=0.02831096138909363, P87=0.03593619499141891, P88=0.084241400869426, P89=0.07251175384836274, P90=0.037148948863804064, P91=0.019373625380627067, P92=0.05304313832944265, P93=0.05511728686357486, P94=0.10981300416090145, P95=0.17561950940253496, P96=0.12801802383198907, P97=0.060987649312483704, P98=0.041734329142586374, P99=0.03497792335578537, P100=0.017466430752318334, P101=0.013606902741521616, P102=0.0066065432323625115, P103=0.00629567184288729, P104=0.00844723074770344, P105=0.009901342283439888, P106=0.008942801429579521, P107=0.008319777713663937, P108=0.005155067970094252, P109=0.010973577394811182, P110=0.014612144425262121, P111=0.022234459611406323, P112=0.02645508712123957, P113=0.012639898943161318, P114=0.013352943624227257, P115=0.016103579222710832, P116=0.020102394186922855, P117=0.00996886070987561, P118=0.005881680929183832, P119=0.004012729225068423, P120=0.004574055777796334, P121=0.003835222237381734, P122=0.004348873541554105, P123=0.00451666947038798, P124=0.006102567829290845, P125=0.0060023990241496695, P126=0.01396114217747689, P127=0.011698538514099377, P128=0.003909593646529561, P129=0.005711672742691948, P130=0.004869282543095383, P131=0.0039933631075534095, P132=0.0036586430124524514, P133=0.003436265000071791, P134=0.0026446062806278763, P135=0.006647868006561795, P136=0.010434623273226177, P137=0.015838591196582714, P138=0.020817547134163294, P139=0.010182508889542908, P140=0.003839812095417699, P141=0.003380900224864533, P142=0.0028641805887680686, P143=0.0038138283817187907, P144=0.0036148403781843326, P145=0.00418443952420189, P146=0.0021418156553767768, P147=0.0015136573530000702, P148=0.0015938499471851283, P149=0.001365446531539203, P150=0.0008437386912569237, P151=0.0009079048493701497, P152=0.0009949918102425785, P153=0.0011742863745462589, P154=0.0011423324629249688, P155=0.002487081951464182, P156=0.0017933152092356375, P157=0.00200313106358958, P158=0.0019480377561745393, P159=0.001622906815800959, P160=0.0020480610794879088, P161=0.0017325260712616208, P162=0.0017426360821985655, P163=0.0015060998471932985, P164=0.0012505084391204453, P165=0.0012865534132576796, P166=0.0007528311891922529, P167=0.0004248647174945389, P168=0.0002810782735179013, P169=0.00027538895605636586, P170=0.0003201193173368436, P171=0.0003026695542063441, P172=0.0002784000271766507, P173=0.00023065492043223613, P174=0.00022966821869187712, P175=0.00018271625182048293, P176=0.0001762702962923414, P177=0.0001831987060244224, P178=0.0001228452056925918, P179=1.5628004027813693e-05),Row( P0=-602.1508204923831, P1=62.233556218042885, P2=-11.967373673797972, P3=10.474754006383867, P4=-7.1586201045453235, P5=-11.099625183316146, P6=4.686215733004543, P7=-7.262013187508872, P8=3.865739425300662, P9=-0.35235457093982775, P10=-9.381222534998779, P11=0.5762005794301459, P12=-2.208674390663141, P13=-5.3392518202959955, P14=-1.903606416258244, P15=-1.7222482821454097, P16=-3.3443960185598804, P17=0.1097640886288499, P18=-1.7356145850612794, P19=0.7137425562409464, P20=-5.873508737062214, P21=-1.4784764192437, P22=-2.854650002914364, P23=-1.1793154374759278, P24=-3.3251155291246746, P25=-0.262494423286976, P26=-4.842558680085602, P27=1.0780971621707172, P28=-1.6317242671289125, P29=-0.8963066816689653, P30=0.12061142903416172, P31=-0.13499305860692395, P32=-2.8046545781790297, P33=-0.37375006564665686, P34=-1.0242790414714842, P35=-1.4602297393062282, P36=-2.0332509495976696, P37=-0.37015911692149656, P38=0.3303692800434018, P39=0.8327081064525417, P40=0.6892996828385334, P41=0.6868211421842245, P42=0.6923457910886556, P43=0.6934381241914125, P44=0.6848340869432951, P45=0.7205879995721832, P46=0.774494475631933, P47=0.7950984381133732, P48=0.7902302877806344, P49=0.7544670320459682, P50=0.719251757240319, P51=0.6966576576276085, P52=2.9368433688070252e-05, P53=0.00010019922797781698, P54=7.917474174957775e-05, P55=0.0001708380788061015, P56=0.0033307138214717243, P57=0.022827309612295732, P58=0.013864336985480428, P59=0.013320339307463652, P60=0.016965231208982568, P61=0.009317946153911351, P62=0.01070897327454587, P63=0.02235574356591586, P64=0.01390979995185746, P65=0.010534312648796318, P66=0.017901316624796736, P67=0.04709191392356151, P68=0.04662841900914175, P69=0.03152665974037427, P70=0.04329051204026382, P71=0.02554675958550847, P72=0.007272046100009659, P73=0.00746651519909782, P74=0.02201933100939066, P75=0.08762146843597836, P76=0.11574727851353503, P77=0.07127204938836838, P78=0.06051032131714327, P79=0.06835466310232205, P80=0.08015374458924489, P81=0.05500055623636511, P82=0.038186440485719855, P83=0.03580898854147327, P84=0.033863742821259424, P85=0.017034909695306764, P86=0.013469168194878032, P87=0.015150215127424982, P88=0.031602538031922194, P89=0.0400078575028314, P90=0.015658902814359457, P91=0.009733891761198642, P92=0.02838347857995723, P93=0.04214208904738695, P94=0.08118703042342867, P95=0.07413995561371636, P96=0.07289516036121403, P97=0.04867718945140711, P98=0.06863733533251541, P99=0.05054724855787583, P100=0.010937082471476225, P101=0.008950423687925596, P102=0.0058777570471714976, P103=0.005079647134831161, P104=0.007239244863174597, P105=0.010052496221297126, P106=0.008619238751090002, P107=0.024501379700540508, P108=0.018966358618188364, P109=0.01275414253656086, P110=0.01883056229318368, P111=0.019273300936636997, P112=0.0159428658025923, P113=0.007785936815777167, P114=0.004980883369413037, P115=0.004819201458210008, P116=0.0025798458207941543, P117=0.00878804088543686, P118=0.009721768751825312, P119=0.010099226178758329, P120=0.012666311020210947, P121=0.009537432084825133, P122=0.005417125158216129, P123=0.0034436465766344406, P124=0.002599152200598174, P125=0.0025236558934709037, P126=0.0042546606916794225, P127=0.0035281245933104478, P128=0.0018263428877396237, P129=0.0007687010240678635, P130=0.0007060596872910794, P131=0.0005166968630579502, P132=0.0007483567149690532, P133=0.003145627848946105, P134=0.0037621498581079755, P135=0.008181257091840269, P136=0.013223483468544085, P137=0.025488111135597815, P138=0.016161747153397462, P139=0.009141262419665612, P140=0.006182077431135857, P141=0.007619749796361876, P142=0.0053004446947216375, P143=0.00453179106985418, P144=0.0033468543829213515, P145=0.003173301276785678, P146=0.00232711126955184, P147=0.0013477851206342402, P148=0.0003019060374803132, P149=0.00029268518412458715, P150=0.000296625489094262, P151=0.00011071306917212413, P152=0.0001254214478988367, P153=0.00012539532910022048, P154=9.905549597143724e-05, P155=0.00010537205697155447, P156=9.446135452108166e-05, P157=8.964504932957775e-05, P158=7.439730066895394e-05, P159=7.906293804866585e-05, P160=0.00014337972330781128, P161=0.0002000808920064682, P162=0.00016027499153284922, P163=0.00014023734772143723, P164=7.071425651934201e-05, P165=4.2585040012059e-05, P166=4.3131482334146416e-05, P167=4.3449696017587474e-05, P168=4.46097398901814e-05, P169=3.9559045091096816e-05, P170=3.279436717002635e-05, P171=3.446545997006285e-05, P172=3.873377965493633e-05, P173=4.992026383589701e-05, P174=5.487963856071094e-05, P175=8.020472420657875e-05, P176=7.186939412315584e-05, P177=4.896289267985137e-05, P178=2.121741411490569e-05, P179=2.178767144963366e-06),Row( P0=-480.8383497631063, P1=46.252478323338224, P2=-25.212543646365713, P3=0.9087653090966923, P4=-19.860527210415334, P5=-18.90064992787219, P6=-6.692930823424814, P7=-10.23788609900292, P8=-7.327266175095567, P9=-4.79870455755596, P10=-16.411016762270606, P11=-4.572106426798194, P12=-8.340474770405466, P13=-9.092696229206974, P14=-1.4252889238466757, P15=-5.86642732118678, P16=-3.6200156071645138, P17=-1.0794350975078377, P18=-2.257550793914159, P19=-0.7008198901891607, P20=-3.5897934443657356, P21=0.263421641981889, P22=2.534734469533497, P23=1.1814445410245393, P24=1.981843389852964, P25=3.414638231295136, P26=-1.4536993785398942, P27=1.6265936569320691, P28=2.839063353275582, P29=-0.14332294239397794, P30=1.1656636208354623, P31=0.4815522469900784, P32=-0.5110101516432308, P33=0.20301517241222233, P34=-0.09692350786477108, P35=0.3579427796589338, P36=-1.7190931617673135, P37=0.29505772720806744, P38=-0.2643322988828285, P39=-0.4897210154458651, P40=0.67026525180718, P41=0.6347748510441467, P42=0.6310011061746593, P43=0.6323079101522969, P44=0.6569385892400644, P45=0.6647986790761836, P46=0.6603896537601112, P47=0.6589877099655334, P48=0.6284156030549242, P49=0.6526777171851347, P50=0.6926058877925437, P51=0.7063845317806613, P52=6.060948159596498e-06, P53=1.5249013471094748e-05, P54=2.518073420814708e-05, P55=6.0250436325678334e-05, P56=0.0007426976640930444, P57=0.005816399115338606, P58=0.007126203417941588, P59=0.02165671495157632, P60=0.06508550317210911, P61=0.19703035949856862, P62=0.38501754901439317, P63=0.3325273977803102, P64=0.11529240658686789, P65=0.1394378752011326, P66=0.19103105050445135, P67=0.18090998113984877, P68=0.2004716147949351, P69=0.3282526922200385, P70=0.3459743957767448, P71=0.22823457090038501, P72=0.28104850978634177, P73=0.358338727652088, P74=0.2682190975730944, P75=0.2237241131264123, P76=0.1752460244497984, P77=0.2288288338130592, P78=0.5188580840319338, P79=1.055814860118114, P80=1.931342117550377, P81=3.6752956955856697, P82=2.262441946859864, P83=0.47984349917956853, P84=0.5894398142960823, P85=0.832124858092446, P86=1.5837632538871411, P87=1.926580016517372, P88=0.5707503799492849, P89=0.4617879407105522, P90=0.3770266618450844, P91=0.32646515335771203, P92=0.4519813356960251, P93=0.23417311992199516, P94=0.10689897482547674, P95=0.9336419082684443, P96=0.8368427612047844, P97=0.284612047958447, P98=0.23476773332863218, P99=0.10551798657276348, P100=0.04166499888346153, P101=0.015382969743690984, P102=0.00976398418764063, P103=0.020180498995691732, P104=0.059030839714910216, P105=0.05176825022564685, P106=0.03268733214095787, P107=0.05493630355177869, P108=0.035224474349923984, P109=0.02350168364789333, P110=0.057355456887026625, P111=0.14864936750538568, P112=0.1032306179408122, P113=0.043210507909758014, P114=0.06027091068344061, P115=0.017351352745643596, P116=0.03125010904442138, P117=0.06870428631891272, P118=0.03787449838915348, P119=0.017138048912763056, P120=0.021272159139065827, P121=0.030723179515196465, P122=0.022678730908715392, P123=0.030493519959428412, P124=0.02538830703155849, P125=0.007614804486633283, P126=0.01317395151975226, P127=0.022357865721371002, P128=0.008494896130172604, P129=0.003702117256856489, P130=0.004720288107674501, P131=0.0032248379789855783, P132=0.0030145554454056177, P133=0.00843038741126897, P134=0.005405043220313641, P135=0.010589594193256186, P136=0.017530450796418625, P137=0.010371303499759359, P138=0.014737639563745002, P139=0.012000411206061098, P140=0.009694995876012302, P141=0.01338981766604943, P142=0.018170417073913654, P143=0.01574918024455117, P144=0.009417288988617097, P145=0.010069476941271714, P146=0.0110941927867716, P147=0.0038854083370489043, P148=0.001842505524587404, P149=0.000744423549793732, P150=0.00044342899096132135, P151=0.0006372440712151332, P152=0.0009220749509863691, P153=0.0015488776907177791, P154=0.003182622402957276, P155=0.005597304646344986, P156=0.003864453236118577, P157=0.004179994858989295, P158=0.00495306488532243, P159=0.003787139358768705, P160=0.004473940691224614, P161=0.006232466988772377, P162=0.004995307915283926, P163=0.004445918411024697, P164=0.0034578462640887713, P165=0.0024481131609343006, P166=0.001469865430463691, P167=0.0010342663696260004, P168=0.0008821177865869953, P169=0.0006566652008611929, P170=0.0004354178046379662, P171=0.00032530367238650475, P172=0.000346255418992913, P173=0.00022251839611929887, P174=0.000206193015986581, P175=0.00016178836339342543, P176=0.00012765186994939364, P177=0.00015162137105423224, P178=0.00010232700472002936, P179=1.7382727590227543e-05),Row( P0=-643.441546953999, P1=85.32401749810234, P2=3.5243584624103876, P3=11.374017845458967, P4=-4.360945910979402, P5=-9.087987765223467, P6=2.375703105901525, P7=3.3746155658721766, P8=4.576379228608587, P9=-2.5109373894445017, P10=-5.246618403473952, P11=-1.5712558655687412, P12=-4.508111336562478, P13=-6.278875692675985, P14=-1.03467826892544, P15=-1.2228665245409485, P16=-3.4548998151103403, P17=0.7236187475660681, P18=-1.4157717641849377, P19=-2.91087010753527, P20=-7.432311033029981, P21=-3.959080281773565, P22=-4.088218224707651, P23=-4.253106751704155, P24=-5.354005171114583, P25=-1.4737813905224881, P26=-7.719779039011613, P27=-2.72664901577592, P28=-3.6428046426161043, P29=-3.0736426293903008, P30=-3.6154938062618984, P31=-2.5928357351465325, P32=-3.565010382870214, P33=-3.7560712283449402, P34=-3.1380773960775317, P35=-1.7795457798654963, P36=-2.1527349675042404, P37=1.1351015828762934, P38=1.726183527111152, P39=2.80394925771897, P40=0.7006812102316222, P41=0.6441417461055149, P42=0.6185206787065358, P43=0.6733580104431247, P44=0.676908261053649, P45=0.6515047921322541, P46=0.695107859774147, P47=0.7077893543454535, P48=0.7313631423361354, P49=0.7518750483941239, P50=0.7209808093837556, P51=0.6747648911777749, P52=7.682930510118746e-06, P53=2.7851432311744286e-05, P54=4.1087279958924066e-05, P55=0.00023211487119695372, P56=0.00771490193313718, P57=0.07124462477118793, P58=0.05974710460069275, P59=0.014434918000537311, P60=0.0026559553545875154, P61=0.0012229197647060776, P62=0.010376114995902692, P63=0.02088698023804342, P64=0.024520974762324774, P65=0.01759562969637652, P66=0.019648263854927604, P67=0.035266283390869485, P68=0.013305975994648488, P69=0.004605614382884505, P70=0.02745309462216805, P71=0.030086096092539174, P72=0.005843543061453482, P73=0.004506394168562165, P74=0.007600732437233864, P75=0.015180141788141789, P76=0.01392146338774153, P77=0.04436477699553503, P78=0.052560982365653146, P79=0.03532613512031586, P80=0.02377516911818417, P81=0.009365455563773857, P82=0.009138813346668221, P83=0.03367601511354811, P84=0.03797783187333157, P85=0.02162212628855774, P86=0.024988578514491655, P87=0.015865511090146448, P88=0.012714136590939612, P89=0.01915914080196656, P90=0.02213512849465719, P91=0.02023057487284282, P92=0.01502400233080759, P93=0.04144864115278952, P94=0.03353295134741919, P95=0.019597387442700308, P96=0.003658286500711154, P97=0.0072057208710718575, P98=0.006536812379797686, P99=0.001813320567120951, P100=0.0011386405097487625, P101=0.0026793884678258757, P102=0.0007697268697379697, P103=0.00018132718063167615, P104=0.0005153615003524815, P105=0.0004041615573919769, P106=0.0005009584523607101, P107=0.0009222347092917613, P108=0.0004808022344783118, P109=0.0012387527317512489, P110=0.0021143763200388503, P111=0.0038079878618578994, P112=0.004848102452121506, P113=0.0010011623104287397, P114=0.0007781598371985449, P115=0.0005261793701736676, P116=0.0006753378312125035, P117=0.0005668762750246724, P118=0.00043761114299578803, P119=0.0002293707966857068, P120=0.00023740843855285453, P121=0.0003716806029464349, P122=0.0002311396616093532, P123=0.00011379899781156829, P124=0.00010808084391390615, P125=7.9583488096667e-05, P126=8.275681642006756e-05, P127=9.636113575280553e-05, P128=4.497295982280294e-05, P129=3.808691981697486e-05, P130=3.373947683195083e-05, P131=3.43007478601835e-05, P132=3.695413121341713e-05, P133=0.00011614175845094467, P134=7.404042595828537e-05, P135=0.00010107893419816424, P136=9.336342862325383e-05, P137=6.381255969845974e-05, P138=4.016054705689368e-05, P139=4.7267863693898315e-05, P140=3.465933032545597e-05, P141=6.211219746747127e-05, P142=9.005019273953669e-05, P143=0.0002346137559726562, P144=0.0003517898537417567, P145=0.00014622816611923752, P146=0.00020644478982085162, P147=0.00016670681995495522, P148=9.305114379028626e-05, P149=0.00011095363008863537, P150=0.00011110747098499793, P151=0.00019321307046897941, P152=0.000302803830098924, P153=0.0005863896892793839, P154=0.0005887497017465427, P155=0.0006008185734097641, P156=0.0009147757630632001, P157=0.00075799471134048, P158=0.0011592407660135668, P159=0.0008585702686456427, P160=0.00021880063721634588, P161=0.0002012681705389266, P162=0.00015885000411513392, P163=0.00013047295068080336, P164=0.00012612157098852437, P165=7.224322569312918e-05, P166=4.9002465171980216e-05, P167=4.7559262394496013e-05, P168=5.807921439279554e-05, P169=5.0955751755262325e-05, P170=4.741382177692762e-05, P171=2.8218502215380786e-05, P172=2.3283299203442923e-05, P173=2.5398078713522113e-05, P174=2.2386903443621626e-05, P175=2.1312193192363425e-05, P176=2.448763896680628e-05, P177=2.5186709486519603e-05, P178=1.429506168093335e-05, P179=2.1240163760503566e-06),Row( P0=-596.9258113535585, P1=61.97516628126855, P2=-17.66841758587015, P3=3.6571214784571593, P4=-13.755042958289641, P5=-15.695772619603474, P6=-5.96451437948873, P7=-3.370668112604398, P8=-2.0885476151372284, P9=-6.890670756734873, P10=-11.097219854871884, P11=-3.7934653413194677, P12=-7.5074172811096025, P13=-6.9282959505580095, P14=-1.718514573619459, P15=-2.954442785350529, P16=-4.362029552907444, P17=-1.8884630862754703, P18=-2.980505412864097, P19=-4.089675434759019, P20=-6.880069423031421, P21=-2.4006309045751295, P22=-2.975594664428513, P23=-3.1881199890435177, P24=-2.46861642399381, P25=0.15896807833683257, P26=-4.297458037973246, P27=-0.7250403116270299, P28=-0.4045718012536403, P29=-1.727722365305993, P30=0.5088574880531258, P31=2.7023828718968717, P32=2.19375582420149, P33=3.8855205819670307, P34=2.6045705299529955, P35=2.132184964587285, P36=-1.4771835649453529, P37=1.7347181718405094, P38=0.7463463260826528, P39=2.525578210756191, P40=0.7577478132119123, P41=0.6719056088927298, P42=0.6211530572469721, P43=0.603754035610942, P44=0.6222287132105155, P45=0.6422768352986712, P46=0.6307677554351936, P47=0.6676370219590166, P48=0.6584075187025409, P49=0.6668408287018495, P50=0.6859589251347524, P51=0.7267877968180835, P52=2.7074336409286757e-06, P53=7.409016551799807e-06, P54=1.5873274491358895e-05, P55=7.976242469326001e-05, P56=0.0017684221010900337, P57=0.020100805922523123, P58=0.031730885307766875, P59=0.04770522059334808, P60=0.03143657080159718, P61=0.07732089718150599, P62=0.10274098998190956, P63=0.03634561469282488, P64=0.014226278004126694, P65=0.031593495412634405, P66=0.1302279344800276, P67=0.3639778781000427, P68=0.17688380244268392, P69=0.021384331073301253, P70=0.03908781040709243, P71=0.043883022953426276, P72=0.06224348984261952, P73=0.08627420833038625, P74=0.10139944688454361, P75=0.11400147821618893, P76=0.0574759731682372, P77=0.022530019048289038, P78=0.03253879865696201, P79=0.04201294066844902, P80=0.039284256730560525, P81=0.05225348726589824, P82=0.11609718464052916, P83=0.20748402477634445, P84=0.14743157873559537, P85=0.051954665095823194, P86=0.05409238095569958, P87=0.04464511488708727, P88=0.03536763300132995, P89=0.05545096777360072, P90=0.10675882330478954, P91=0.15978032804328748, P92=0.1526741626109085, P93=0.06584343245599951, P94=0.05463388274329433, P95=0.16681279912441102, P96=0.18241093751817905, P97=0.0697525476588952, P98=0.05232848602735373, P99=0.033807705066139976, P100=0.0078275878305617, P101=0.004938305743964665, P102=0.0031863610196903548, P103=0.006621155943266051, P104=0.005161440402560825, P105=0.005403487254393286, P106=0.005787366411163663, P107=0.008479124086298133, P108=0.0076427447760460294, P109=0.011350264807092584, P110=0.018044123816545462, P111=0.02368241367995045, P112=0.01917775934126244, P113=0.017807956775463294, P114=0.009368270132006383, P115=0.0031183155720376113, P116=0.005390272118052896, P117=0.00429330646098625, P118=0.0051453173206736725, P119=0.002349273345709642, P120=0.0017846701127091236, P121=0.0022672838658431196, P122=0.00197488762384353, P123=0.0025730371013481363, P124=0.0021399859817845, P125=0.002616079798353757, P126=0.0017925987872875438, P127=0.0013568773820805896, P128=0.001057445617800036, P129=0.0003490922783076205, P130=0.00024197256789265073, P131=0.0002103565191760696, P132=0.00021668306545168683, P133=0.0006715293822756367, P134=0.000671974151482952, P135=0.001484811198864793, P136=0.0025698164682643617, P137=0.0019509832102925957, P138=0.0006444941879978049, P139=0.0005822649514495554, P140=0.0005021719215847251, P141=0.0005389416725885732, P142=0.00063341440096819, P143=0.0006860116919851612, P144=0.000900851837817603, P145=0.0004045476949445923, P146=0.0004370345409614991, P147=0.0004184438248822228, P148=0.0003451723208207553, P149=0.000186201895411628, P150=0.00028549496355916325, P151=0.0005835982213564613, P152=0.0006564542920656155, P153=0.002395674268461887, P154=0.002727699010679651, P155=0.001890836210612625, P156=0.0038314639135683744, P157=0.00348075321984668, P158=0.002122459711173815, P159=0.0032751093850117677, P160=0.002968528288176489, P161=0.0028538203965092256, P162=0.0022746401740495283, P163=0.00144482715675964, P164=0.0007113349345015076, P165=0.0010302727086374578, P166=0.000777763078505794, P167=0.0007791900078554628, P168=0.0005094467948478322, P169=0.0003940820397270738, P170=0.0002533895146885251, P171=0.000203027812390749, P172=0.00014342835874562695, P173=0.00013796338345359953, P174=0.00010526095753339238, P175=8.649559579310693e-05, P176=5.610115059232969e-05, P177=6.168868335740248e-05, P178=5.0004314756113464e-05, P179=4.696184712004721e-06),Row( P0=-591.3860415983611, P1=59.842550807201995, P2=-15.323044161758947, P3=5.854904594555085, P4=-14.658977992872307, P5=-15.096996064612286, P6=-3.3787418777847638, P7=-8.269508460875137, P8=-1.2349593252003688, P9=-3.9954189700659932, P10=-10.71172703531617, P11=-1.6611830033549488, P12=-5.793288985496408, P13=-9.515648856679826, P14=-1.1837096513686387, P15=-2.048340471218201, P16=-3.35190033505598, P17=-0.8038814968841452, P18=-2.9348291605574537, P19=-1.6446113528158801, P20=-6.905160444999068, P21=-2.149395407165321, P22=-3.390042051473955, P23=-2.373792735165857, P24=-2.657401359475147, P25=1.0768760181318093, P26=-0.7639516371759085, P27=3.7938335039072357, P28=0.7413734525662231, P29=-0.08160131127449244, P30=-1.6177199946896104, P31=-2.0242175305985093, P32=-3.40833830467499, P33=-0.08835765129671656, P34=1.265357270295114, P35=2.009674004810109, P36=0.9478907688592659, P37=2.6395844268943285, P38=1.231992393414993, P39=1.958189593602783, P40=0.7077772979457797, P41=0.6473911950711322, P42=0.6448066580640542, P43=0.6578888841379725, P44=0.683633210085939, P45=0.6744170351091545, P46=0.6615304370698155, P47=0.6698054949364625, P48=0.705599907485428, P49=0.7547179518963597, P50=0.7919001332490383, P51=0.7662019641102856, P52=4.557869010150211e-06, P53=1.3025622117513637e-05, P54=2.530766693001004e-05, P55=8.32273906417793e-05, P56=0.0015238806368443796, P57=0.011874768470118174, P58=0.017452461167103066, P59=0.01870304378150765, P60=0.05378665895986644, P61=0.12445282242069065, P62=0.07711972044234618, P63=0.024995620309426946, P64=0.01822546901114134, P65=0.05726738693999715, P66=0.0689974331037735, P67=0.052388601030881575, P68=0.03624293282974813, P69=0.08196579294725785, P70=0.18689613867232702, P71=0.2111145614568355, P72=0.0997075980038489, P73=0.05960289141573056, P74=0.03090599203391983, P75=0.019057892274326707, P76=0.016703903757863126, P77=0.036266285725634256, P78=0.08047636556741282, P79=0.14751645670942823, P80=0.19706820147893803, P81=0.22750648223565656, P82=0.164049317029506, P83=0.049355637493317096, P84=0.04209072585837143, P85=0.047132276396500444, P86=0.048022879217110036, P87=0.039518813059133684, P88=0.05805809225855495, P89=0.04004241016920977, P90=0.05706926988582441, P91=0.06314410489400493, P92=0.05952516221101208, P93=0.012554195389447445, P94=0.007126583927844786, P95=0.0050321993549814845, P96=0.03285675017792258, P97=0.1324654015108003, P98=0.04916327397932914, P99=0.0048461411347208395, P100=0.004634164663095276, P101=0.004926177589285968, P102=0.0037834674244404867, P103=0.011043218154101402, P104=0.0026814056085910968, P105=0.0013271502661509006, P106=0.0017081863340443618, P107=0.007446580782449816, P108=0.005812746253405514, P109=0.002682204582590121, P110=0.0067192305393486895, P111=0.03679924827891826, P112=0.05268469121609929, P113=0.003061917534166391, P114=0.0005575107776047335, P115=0.002274633418917179, P116=0.001707582758177342, P117=0.0027881806139367026, P118=0.005337493813523587, P119=0.0020037236211072136, P120=0.0036620953868275767, P121=0.005742502314146542, P122=0.002552729972047722, P123=0.007886404233135318, P124=0.00882065128395074, P125=0.0018389288168827306, P126=0.0017224029562731818, P127=0.0008034433876015412, P128=0.000862654783338912, P129=0.0005864118857177222, P130=0.000632482999507017, P131=0.0003821690177840181, P132=0.00033792418479398185, P133=0.0009888746702497926, P134=0.0021607148794188682, P135=0.005112752635569299, P136=0.007166702325478036, P137=0.0022131980865222824, P138=0.0018882785097728626, P139=0.002783707798789144, P140=0.0029520022591017917, P141=0.0038103607839967317, P142=0.0014676787490854736, P143=0.0010125006677378964, P144=0.00111519339909029, P145=0.001229920787750337, P146=0.0009928026915697235, P147=0.0004966737747381338, P148=0.0001927144706200819, P149=0.00013514389638130195, P150=9.630162612717342e-05, P151=0.00011062978411950977, P152=7.516676708309708e-05, P153=8.565671794216912e-05, P154=0.0001347649101226026, P155=0.00014068218509899176, P156=0.00010689915877628796, P157=9.628936706336126e-05, P158=9.713940926704563e-05, P159=0.00018750763802963868, P160=0.00017495025296439625, P161=8.755109769179858e-05, P162=8.115458450619678e-05, P163=0.00013364909583124236, P164=0.00010926384770914595, P165=8.138328283117527e-05, P166=6.193995418609557e-05, P167=5.585462788711393e-05, P168=4.582503255417519e-05, P169=4.2085302555943936e-05, P170=5.0162996824742675e-05, P171=4.4708863171968255e-05, P172=4.090849606833006e-05, P173=3.673412456192102e-05, P174=3.024304558821653e-05, P175=3.156066010820382e-05, P176=2.7170039568107958e-05, P177=3.06235243769962e-05, P178=1.7892003474780435e-05, P179=1.8200263678243045e-06),Row( P0=-587.3366428454568, P1=62.92835039897951, P2=-14.473176325831618, P3=9.093285696937137, P4=-9.461219820307043, P5=-15.340699135006464, P6=0.3591171423930143, P7=-5.140492989913158, P8=-1.328441797576679, P9=-2.4119972834423065, P10=-11.01792550587994, P11=-0.9711223958611935, P12=-5.311226421206317, P13=-6.466368234358902, P14=-0.6839940232978416, P15=-3.810664535504421, P16=-4.548325526642753, P17=-1.1225938212972257, P18=-2.7304540675786724, P19=-3.473612037149979, P20=-5.732047623474811, P21=-3.005016801654607, P22=-1.5304081815760007, P23=-1.1189973327188711, P24=-1.3808421612682995, P25=0.9208408594431393, P26=-3.80630644285325, P27=-1.1712570882472564, P28=-0.6836683738590547, P29=-0.13180113587869804, P30=1.6394190711660532, P31=2.818502948330698, P32=1.8596774261137652, P33=4.000547855329515, P34=1.4854071606202832, P35=1.1452159669910178, P36=-2.1135028386347945, P37=0.5272324865139496, P38=-0.18076895460845305, P39=2.042250021319141, P40=0.7435187627877937, P41=0.6608528511520044, P42=0.6114649028068045, P43=0.6401564048498162, P44=0.6584651400351921, P45=0.6457332753450484, P46=0.6800977179561654, P47=0.7076336388953527, P48=0.7050290850268044, P49=0.7243344647906567, P50=0.697757907342323, P51=0.6986641745358979, P52=9.550558461640329e-06, P53=3.291118850323578e-05, P54=5.067813790095421e-05, P55=0.0001911796822565628, P56=0.0020203097851985677, P57=0.01766659365539313, P58=0.025624625864369803, P59=0.05590610181947339, P60=0.05270653624122617, P61=0.06344582127903409, P62=0.0779624303869773, P63=0.05689469998017151, P64=0.023360844088156467, P65=0.021511068519909426, P66=0.03435281314607556, P67=0.15210557156970636, P68=0.10883774165679552, P69=0.04141152945395345, P70=0.06549249331602851, P71=0.07311239908088092, P72=0.05219434017074324, P73=0.04830360530415663, P74=0.06562076755102367, P75=0.04319748280629977, P76=0.045710092635672106, P77=0.08914801349253228, P78=0.11255226504005529, P79=0.10718952127374866, P80=0.08039209352339696, P81=0.054581912440854134, P82=0.05270839152103223, P83=0.07026974454993266, P84=0.09072357533679887, P85=0.06780960381766898, P86=0.028223862370531896, P87=0.007147267231435642, P88=0.011160170667956599, P89=0.04199364074288841, P90=0.09491588098045257, P91=0.049630826131162466, P92=0.008655326185273345, P93=0.015018060838861853, P94=0.1550814178126447, P95=0.4166080745213757, P96=0.31470582092906296, P97=0.028177500269407216, P98=0.08198338607032404, P99=0.04882558481091481, P100=0.010580415301578924, P101=0.007228390819272285, P102=0.0018307491195525935, P103=0.002243491428368156, P104=0.008186756889961929, P105=0.011300161852433847, P106=0.0024047602351766713, P107=0.003959889911606065, P108=0.016341766677102694, P109=0.006701818892931713, P110=0.006601752351479089, P111=0.05586400416144252, P112=0.05396922643056104, P113=0.022089554829641925, P114=0.00844563594381177, P115=0.009124297870429685, P116=0.011939089908035737, P117=0.0055612889016171405, P118=0.006999042771329192, P119=0.012605117993422798, P120=0.003955325675891861, P121=0.011097505278588103, P122=0.0049465349870067175, P123=0.006194057659188901, P124=0.00296339481170989, P125=0.0012619785116476682, P126=0.0012345819526822593, P127=0.0022158023148433907, P128=0.0014978970196455237, P129=0.0013769445191809805, P130=0.0007054595442747116, P131=0.0006316831150201278, P132=0.0008507479884190728, P133=0.0035250330258286885, P134=0.0020512497536996194, P135=0.002662462649974442, P136=0.0023700425390979917, P137=0.0016468489239042645, P138=0.0013745928161205251, P139=0.0013777347717064653, P140=0.0012605901963272, P141=0.0015377404579175772, P142=0.001141354254506456, P143=0.000780482029396074, P144=0.00074037504608906, P145=0.00037254778024529945, P146=0.0003385165143019876, P147=0.00029110078824632057, P148=0.00016181352401523235, P149=4.4135923198989894e-05, P150=3.3769364855844804e-05, P151=4.324522939706171e-05, P152=6.525067750402108e-05, P153=0.000158892920344195, P154=0.00021593892777441994, P155=0.00027128443388589846, P156=0.00018601757055347438, P157=0.00020668708967075996, P158=0.00015749907665712175, P159=8.142479280889432e-05, P160=0.00013870442410421889, P161=0.0002009661045757567, P162=0.00023601955776279251, P163=0.00021501852899022243, P164=0.00012527951562835946, P165=5.68752040449086e-05, P166=9.144886514119261e-05, P167=0.00011614455837986115, P168=7.992660038065266e-05, P169=7.472660923735077e-05, P170=6.768655569453196e-05, P171=4.267700745877667e-05, P172=4.73668579663972e-05, P173=3.377388586731666e-05, P174=4.019827965065206e-05, P175=4.493434999789439e-05, P176=3.714029937973532e-05, P177=3.986678933967412e-05, P178=1.8429335725937355e-05, P179=2.591622025761759e-06),Row( P0=-593.5960952322497, P1=60.75621935672245, P2=-13.043995689989167, P3=4.233607195941618, P4=-11.778789937486447, P5=-13.53688808879242, P6=-2.366780276905956, P7=-6.431970536697339, P8=-2.326713928955887, P9=-4.5286375062298925, P10=-10.441083412021527, P11=-1.946835259626887, P12=-6.037410203692718, P13=-6.234894840965502, P14=-2.420484163217336, P15=-5.120310605132796, P16=-4.544773534348188, P17=-0.9475170277121762, P18=-2.750494686387705, P19=-4.115307024073629, P20=-6.20558541786325, P21=-3.596268892270532, P22=-3.9049096040738123, P23=-4.225263531357296, P24=-3.025938394676703, P25=-1.3781533232596328, P26=-3.1142172769794105, P27=0.6263587871383354, P28=0.08134813671388896, P29=-1.4231944072080143, P30=-1.1418297661357586, P31=-1.6931924714477253, P32=-1.698744662567019, P33=0.9577235947987545, P34=2.2998882782137655, P35=4.054720074059386, P36=2.844735163158115, P37=4.254017511651944, P38=1.0124954872187426, P39=1.210434940849834, P40=0.6741845371217738, P41=0.6199647237731238, P42=0.6634767062591811, P43=0.6584055873745286, P44=0.6313131901191431, P45=0.6706560078374478, P46=0.6409470491959035, P47=0.6381559324095603, P48=0.7221419956730817, P49=0.7450765369698609, P50=0.7922087022269324, P51=0.7449993077467155, P52=4.187906231523815e-06, P53=1.0685878324126443e-05, P54=3.277588286374513e-05, P55=0.0002082440668637942, P56=0.002677130816601981, P57=0.03232897985361509, P58=0.04583485190681342, P59=0.03385045471071311, P60=0.04376498684537354, P61=0.05949089346578123, P62=0.030479267463612274, P63=0.015004386166689063, P64=0.023055014157666108, P65=0.055736788219076794, P66=0.12859680633992057, P67=0.049617183899006045, P68=0.028710648578523134, P69=0.09590483629516827, P70=0.145333336525169, P71=0.1038577330947779, P72=0.03467618841110683, P73=0.04639669742515548, P74=0.036345382283148084, P75=0.011738313685861255, P76=0.014894377603658885, P77=0.10361363596740168, P78=0.25582579602380323, P79=0.18370400094665704, P80=0.10913666683240299, P81=0.16871269619667473, P82=0.11261837834580249, P83=0.03424689296331172, P84=0.021848130189307563, P85=0.006611741868740182, P86=0.008736760414842453, P87=0.03981653172314203, P88=0.09896590643717187, P89=0.046904894133087204, P90=0.015359326131966482, P91=0.009050498742201507, P92=0.033968299511176886, P93=0.1266698451161998, P94=0.14023154998162743, P95=0.05495651879113923, P96=0.017162114997272243, P97=0.06578301412744023, P98=0.16958234764942445, P99=0.039781504801755053, P100=0.0013202098198748572, P101=0.002623215159803929, P102=0.011233955043660489, P103=0.005045830205199021, P104=0.0025236863506815435, P105=0.0025686327878348403, P106=0.004223840550404967, P107=0.003930742650713081, P108=0.0018655239992505371, P109=0.025005542292494135, P110=0.011215455467598966, P111=0.004315556983884022, P112=0.01561238509410598, P113=0.0108846440905934, P114=0.0041672330829400145, P115=0.005268716767441713, P116=0.001387719635713463, P117=0.003607515743532097, P118=0.003107961183164778, P119=0.0009863621877399039, P120=0.0011181999305893832, P121=0.0014742015148775881, P122=0.0011589422863706935, P123=0.0014020127674904174, P124=0.0014466313552758759, P125=0.0007355506839803328, P126=0.0013976036027160816, P127=0.0035875165841252523, P128=0.0024045133775470453, P129=0.000692185368558726, P130=0.00027827153166636113, P131=0.00021453415001609317, P132=0.00015262983911665145, P133=0.000554214181585436, P134=0.0004533113878886669, P135=0.0006853075425354272, P136=0.0009887175299167668, P137=0.0009127313262085004, P138=0.0009167727010561566, P139=0.0006327542722337187, P140=0.0004414658379989153, P141=0.000562736340448529, P142=0.0007477904627835019, P143=0.0007859283694588357, P144=0.0006929638702736727, P145=0.00022133516283267117, P146=0.0002938618305978586, P147=0.0002874662304415753, P148=0.0001463411454545707, P149=4.642226790829093e-05, P150=3.295824181120397e-05, P151=3.9087158785270855e-05, P152=9.438957111485207e-05, P153=0.00024206348915974566, P154=0.0002301894543695557, P155=0.0002795734410945687, P156=0.00016992489864375322, P157=0.00011621670587179976, P158=8.53478176594874e-05, P159=9.691617879760128e-05, P160=0.0001604510802704499, P161=0.00014073003292640394, P162=0.00013490021984463224, P163=0.00012537340737072322, P164=6.909343301545323e-05, P165=6.663671965920054e-05, P166=4.97108247522195e-05, P167=4.8766372777694916e-05, P168=7.126707974595732e-05, P169=7.660994879051726e-05, P170=4.78399728104075e-05, P171=2.9585017438510687e-05, P172=2.298120186467309e-05, P173=2.7824815854099092e-05, P174=2.5487461562162932e-05, P175=2.4731362743706916e-05, P176=2.163385036501695e-05, P177=2.2240624106280345e-05, P178=1.6985226450662527e-05, P179=2.2101971932838944e-06),Row( P0=-579.2221853043501, P1=63.00455066813203, P2=-12.630494391913047, P3=7.877008790395497, P4=-14.564769513316381, P5=-14.384515468709017, P6=-3.53532654247686, P7=-7.1856710276546405, P8=0.7353783171927921, P9=-2.3203465505459855, P10=-10.254670104233693, P11=-0.32735081353637263, P12=-5.367985128055908, P13=-9.819037941453113, P14=0.010789355063019868, P15=-1.8956091113821942, P16=-4.49125528313095, P17=-0.09965075418241598, P18=-2.801434927347661, P19=-1.6939251245350098, P20=-5.788413180749701, P21=-2.6994948392694007, P22=-2.9036716812109846, P23=-1.352912023326422, P24=-3.5921361475379494, P25=-1.7012676705025709, P26=-3.8054351540754476, P27=1.0125676508181096, P28=0.6849099825645484, P29=0.6710719238338242, P30=1.1684865796573505, P31=-0.26341184959357106, P32=-3.4713484273830737, P33=-1.106060864874046, P34=-0.3321568742405464, P35=2.3260731093932776, P36=1.7926670892571006, P37=3.6739073189825384, P38=1.7513505329776864, P39=1.428773369097456, P40=0.6548280214709771, P41=0.650487139524438, P42=0.6786577669657335, P43=0.6221177468667428, P44=0.5698948584441261, P45=0.6191643132797833, P46=0.6471677779741637, P47=0.6908231328444577, P48=0.7591751181395942, P49=0.7879173237321052, P50=0.791769266731864, P51=0.7047881034683988, P52=5.937965969398021e-06, P53=1.5253859032187573e-05, P54=3.0174998307729163e-05, P55=0.00011314465674374699, P56=0.002268629580931015, P57=0.018722041955077345, P58=0.023883725056097625, P59=0.026793242228540272, P60=0.04899350268049649, P61=0.04144997563527318, P62=0.024586301291849386, P63=0.018385394661972452, P64=0.0182812773072789, P65=0.08908120739692488, P66=0.11904707953534382, P67=0.07363548539006687, P68=0.0616408500775341, P69=0.12383220272957289, P70=0.11401417739616232, P71=0.052930003896768316, P72=0.03278137081548339, P73=0.05264483636927906, P74=0.04110230464429753, P75=0.024456229206067635, P76=0.03696678976730708, P77=0.20752000811583873, P78=0.32948551408596344, P79=0.28588660568952445, P80=0.24114509919995794, P81=0.11699236148708785, P82=0.05512752536605068, P83=0.030435630895626287, P84=0.036502401442513516, P85=0.03296385816325381, P86=0.07152645147494766, P87=0.09339381542769702, P88=0.056733684626278, P89=0.02592155831424699, P90=0.014459430666174552, P91=0.01677095125998608, P92=0.06858130463575318, P93=0.04900042435785243, P94=0.026334184101109458, P95=0.03958140614733146, P96=0.0658581004963925, P97=0.04637833666770475, P98=0.020877669328468226, P99=0.008414405166452968, P100=0.0032377326210857784, P101=0.003919452707822538, P102=0.002993595229331047, P103=0.002610463656154445, P104=0.001868023651104661, P105=0.00483383337678569, P106=0.007018497891603082, P107=0.010304779418296764, P108=0.004170655408669103, P109=0.014260840359277657, P110=0.023803092140127272, P111=0.030223215894162582, P112=0.006308391208675887, P113=0.0033851048535797454, P114=0.0020295046627044422, P115=0.0011443048284892046, P116=0.002002856242916563, P117=0.002648704957014232, P118=0.005182132258332606, P119=0.004549581217059761, P120=0.004416447471622151, P121=0.007056897651632768, P122=0.0026949448625698627, P123=0.003037422553005905, P124=0.0015316949909896624, P125=0.0029381811257900583, P126=0.0009620691813290275, P127=0.0014188170125441612, P128=0.0005867599449988745, P129=0.00040834937659474434, P130=0.00017937993607937722, P131=0.00017300750634506747, P132=0.0002299407031674111, P133=0.0006098542987597039, P134=0.0009495569960890572, P135=0.002760731264079295, P136=0.0038636678355274145, P137=0.001739519446124863, P138=0.001572003263180187, P139=0.0022743227407563595, P140=0.0039633114309839314, P141=0.0031162092622441634, P142=0.0017916939991320729, P143=0.001198149492817984, P144=0.0010288709021617688, P145=0.0009112719395272333, P146=0.0007027700612433222, P147=0.00024329179932099606, P148=9.750974495189853e-05, P149=0.00011011403789108342, P150=0.00011540083849371229, P151=0.00019485252718308427, P152=0.00015111243049705647, P153=0.00012725839112463208, P154=0.00012242276749454244, P155=8.689946523271483e-05, P156=0.00023684757639145983, P157=0.00011635336853223393, P158=0.00012072052027543376, P159=0.00019492969173060028, P160=0.0002032272926550441, P161=0.0001311826229245459, P162=0.000168001325729321, P163=0.00010439258764288368, P164=8.803656188134262e-05, P165=6.196226646773544e-05, P166=4.9328027009524316e-05, P167=4.205174668548202e-05, P168=4.075715785957796e-05, P169=3.171571388115674e-05, P170=3.5502112176152014e-05, P171=2.857049827968573e-05, P172=2.1526365922055664e-05, P173=1.9907742584565566e-05, P174=1.6295988968230604e-05, P175=1.5818848295114874e-05, P176=1.691662468802854e-05, P177=1.2719443363181378e-05, P178=6.252041289738042e-06, P179=1.0008130798708995e-06),Row( P0=-653.2112135759209, P1=85.4091182955791, P2=9.546260439414121, P3=11.438998734605557, P4=-1.9192572971824002, P5=-5.080500641791918, P6=3.577383968909627, P7=2.1707937175987664, P8=8.898534401276432, P9=0.48451668016538985, P10=-3.534389109678588, P11=-1.41478314669191, P12=-2.759487874849696, P13=-4.0646280770004175, P14=-1.291651441689419, P15=-1.1455623319055035, P16=-2.8326510167285956, P17=0.0915891835883777, P18=-0.796944846222225, P19=-1.9326580467555214, P20=-5.271307823596383, P21=-1.6815363881793135, P22=-3.16664637502548, P23=-3.04003303866328, P24=-3.604580031081205, P25=-0.30360016594085987, P26=-5.582847481382994, P27=-1.316847853799436, P28=-2.013271158410906, P29=-1.64117858721185, P30=-1.8031305368430608, P31=-0.6188416280058766, P32=-2.747363618387961, P33=-1.310214654770236, P34=-1.0876192247029894, P35=-0.6045891634644965, P36=-2.124618602829593, P37=-0.5771132450419183, P38=0.47882471003035093, P39=1.027182933565597, P40=0.6940749894010099, P41=0.6658670066503019, P42=0.6942819725375001, P43=0.6849183148224351, P44=0.6873622773238959, P45=0.6963581762576638, P46=0.7336868324373894, P47=0.7491733435121216, P48=0.7442018142393771, P49=0.7029552763169289, P50=0.676179700202901, P51=0.7141766274142287, P52=3.36312790436548e-05, P53=0.00012179254963626447, P54=7.271422182577396e-05, P55=0.00040359767604211623, P56=0.015157003707472002, P57=0.10331294742391806, P58=0.052305455941534365, P59=0.016370493021095407, P60=0.005961161394511369, P61=0.0024216496393136603, P62=0.016881033513665957, P63=0.05084334151816078, P64=0.03093885519170111, P65=0.013558405143871945, P66=0.01650307744965599, P67=0.015349378812621309, P68=0.01560066102355011, P69=0.014637044245991115, P70=0.017016369703124982, P71=0.007713748701006199, P72=0.0037385462027478516, P73=0.00671736479971028, P74=0.011120259853294607, P75=0.013270761936847016, P76=0.028492333980122863, P77=0.0354653563915643, P78=0.02850496970943118, P79=0.02169244085633679, P80=0.019574203709369287, P81=0.03215194319675623, P82=0.044270362373729194, P83=0.024781610706266347, P84=0.015619401183914323, P85=0.01580126988718879, P86=0.02496557528128961, P87=0.03441979162146527, P88=0.03696636780870603, P89=0.020126090107519392, P90=0.03582711505769624, P91=0.04492866666375895, P92=0.03352512253485922, P93=0.007524544225585843, P94=0.023302234521863426, P95=0.0479333900112653, P96=0.01818313834626083, P97=0.002145217223917458, P98=0.0025730689859380677, P99=0.0026047498594547686, P100=0.0012038515318041401, P101=0.0009998292895265777, P102=0.000585199404166914, P103=0.0004718464677026231, P104=0.00046080798982798627, P105=0.0006419423121089515, P106=0.0011470417592966914, P107=0.0015463887588027935, P108=0.0006628899256920605, P109=0.0020631918202565704, P110=0.004525330602933856, P111=0.0041996968570781265, P112=0.003683811819238453, P113=0.0014012208792274182, P114=0.001241993404670935, P115=0.00033883959934281134, P116=0.0008909124519352074, P117=0.0004334262767194204, P118=0.0003953617186012861, P119=0.00031508576180138976, P120=0.00018156788366254019, P121=0.00031982493010549075, P122=0.0001460648831957721, P123=0.00018453752287594014, P124=0.00014797816322465954, P125=9.366382270069815e-05, P126=7.051597180959991e-05, P127=4.041389073945555e-05, P128=2.0386634995673062e-05, P129=3.0403324805502285e-05, P130=3.106866075541415e-05, P131=2.308649400891273e-05, P132=4.215095720884253e-05, P133=8.046081716305994e-05, P134=4.709757197533122e-05, P135=7.457224683245589e-05, P136=0.0001432967989031869, P137=0.00014546989418451556, P138=4.534253567719808e-05, P139=3.644261324277095e-05, P140=3.0233739331120027e-05, P141=6.434528273436634e-05, P142=0.00013701052765433694, P143=0.0003578775774456736, P144=0.00042016367532848723, P145=0.00023040138380561878, P146=0.0002318621955170995, P147=0.0002389775234331567, P148=0.00010637235660596481, P149=0.00016157151715598582, P150=0.0001919243920313502, P151=0.00031221165660303167, P152=0.00040456587789209367, P153=0.0002748146379241129, P154=0.00043464170370833985, P155=0.0006689024109302281, P156=0.0006852532190254974, P157=0.00039277455531934566, P158=0.0002638492481371383, P159=0.0003379277582008359, P160=0.00031334662872986905, P161=0.00018038093621186555, P162=0.00017067709128833734, P163=8.881770991238697e-05, P164=6.262301221397953e-05, P165=3.938580404379292e-05, P166=3.2879030872010435e-05, P167=4.059188447416876e-05, P168=4.3135478355574233e-05, P169=3.417540337114861e-05, P170=2.1081247106992742e-05, P171=1.8543708314870338e-05, P172=1.9749110008140266e-05, P173=1.621105219228738e-05, P174=1.5238192298792442e-05, P175=1.380146738067116e-05, P176=1.8226159719300084e-05, P177=1.9036997465170348e-05, P178=1.556525884704669e-05, P179=1.7579630647311144e-06),Row( P0=-607.6715886042016, P1=65.42639049966905, P2=-12.023287772798989, P3=10.742403243598966, P4=-4.371876809129903, P5=-12.612773823660875, P6=6.060462137373879, P7=-3.210895699790555, P8=4.175869551198712, P9=-3.2630860768398136, P10=-9.534879797782844, P11=0.193263978189092, P12=-3.6867100465753864, P13=-7.656464846275646, P14=-3.0135630793765866, P15=-1.1055679303748642, P16=-4.361405312866517, P17=-1.074779505870933, P18=-2.0941676164864504, P19=-0.34109989588975065, P20=-6.973407161393515, P21=-1.5328853738962338, P22=-4.3003357668705435, P23=-1.9089804376830282, P24=-4.592942073629232, P25=-1.3483367622884226, P26=-5.441406474257097, P27=0.27549386332618986, P28=-2.219856115977201, P29=-0.6152238045036509, P30=-1.0863958549420298, P31=-0.9335912504764988, P32=-3.2000438546570398, P33=-0.49929456161353514, P34=-1.2723282304599624, P35=-1.3637895956393988, P36=-1.1812254382564042, P37=-0.4164745513370109, P38=-0.8991818600269763, P39=0.5092264400053019, P40=0.6921216222189321, P41=0.6570805790666917, P42=0.6562112405116406, P43=0.6779133359648909, P44=0.6537402136522418, P45=0.7140230618475454, P46=0.7861562451638449, P47=0.80533160861895, P48=0.7996013911775285, P49=0.8064390985118955, P50=0.7497871160395833, P51=0.6933077507658406, P52=7.355307462341112e-06, P53=2.5262371919704167e-05, P54=4.346745137284406e-05, P55=0.000178404885951893, P56=0.004102561626297968, P57=0.02049748345950214, P58=0.013216915227366786, P59=0.01702539618593888, P60=0.016364132638555996, P61=0.006578657627679383, P62=0.01138524053699681, P63=0.018836382384587912, P64=0.01110290806248357, P65=0.012010575873052449, P66=0.03664674327779601, P67=0.08459926033881676, P68=0.05059474489606597, P69=0.024569020523618656, P70=0.01737641873996087, P71=0.008270725721625662, P72=0.007247093939017981, P73=0.010998936693791993, P74=0.031608916899048774, P75=0.05766201532309774, P76=0.042353664344825924, P77=0.046761926564322535, P78=0.06560396816222148, P79=0.0661773923930272, P80=0.048848395227502325, P81=0.03505646276829411, P82=0.02996224165490937, P83=0.03124218385583606, P84=0.03028933136410241, P85=0.013405473237108743, P86=0.006519549413887626, P87=0.009565716508816659, P88=0.028674997498506737, P89=0.024722910970466864, P90=0.01322211406783973, P91=0.008861151645792419, P92=0.025634429374747216, P93=0.03522521203573554, P94=0.029536188880360362, P95=0.03508537196404321, P96=0.034083224220235464, P97=0.03767910637063974, P98=0.027771034279901373, P99=0.026756884987365826, P100=0.013280112597041831, P101=0.009084356319094896, P102=0.0038161194333343846, P103=0.0031959260422216194, P104=0.004484229570331284, P105=0.003627347652789796, P106=0.0030683115609644986, P107=0.004258386426028989, P108=0.0036744381857129705, P109=0.0067038324035343455, P110=0.015659782848181227, P111=0.022078492409997912, P112=0.010908430022941855, P113=0.007621449039296919, P114=0.010707487948109336, P115=0.006445171137754544, P116=0.006115459170891838, P117=0.006418209898732069, P118=0.005686425077475995, P119=0.004856536761352703, P120=0.005005774282474367, P121=0.014298069224183868, P122=0.008385140537983365, P123=0.00970989462672979, P124=0.0025632131134759205, P125=0.0033609138614165367, P126=0.004138905244345858, P127=0.01965955606083115, P128=0.005429209275975318, P129=0.003057021415082874, P130=0.0014521791167211376, P131=0.001237752755453351, P132=0.001450501495100249, P133=0.0032758707887417626, P134=0.0034903710236129596, P135=0.011089207253479817, P136=0.013581141068846949, P137=0.01749756500401613, P138=0.021603143616842794, P139=0.011424299160883571, P140=0.006479767377880397, P141=0.004415808911056334, P142=0.0021593667133526966, P143=0.0012055391798156344, P144=0.0010883796958321313, P145=0.0010956008565629364, P146=0.0012632118626103393, P147=0.0012877765094542397, P148=0.00031679379189561527, P149=0.00018682114537644918, P150=0.0002277509206228263, P151=0.000346520986942087, P152=0.0002856660553506095, P153=0.0002738359785894342, P154=0.00017577321690095894, P155=0.0003462051474588256, P156=0.00025097565526758435, P157=0.000127792991335981, P158=0.00014142912947459743, P159=0.00015744803170403236, P160=0.00013604788238174407, P161=0.0002672855130080596, P162=0.00016283025550359795, P163=0.0001590741506027955, P164=0.0001049301084802719, P165=8.107278599507904e-05, P166=7.021025520750249e-05, P167=6.442369641654892e-05, P168=5.455601115245921e-05, P169=9.325773276333926e-05, P170=7.85382438559494e-05, P171=5.839282254246512e-05, P172=5.816954309783092e-05, P173=6.534166806444774e-05, P174=6.946654790959808e-05, P175=6.710101325312095e-05, P176=7.739254203121072e-05, P177=7.993987842400582e-05, P178=4.826942959121367e-05, P179=5.10347694340292e-06),Row( P0=-601.2834817133634, P1=54.9009989142867, P2=-11.233465815577862, P3=6.690373817600441, P4=-9.586792426804516, P5=-13.233399577339073, P6=-4.519806314976072, P7=-5.596192629402562, P8=0.06878912896392687, P9=-3.840990623989642, P10=-10.071699163184972, P11=-1.719265342719137, P12=-8.453518881933826, P13=-7.8749596484074855, P14=-0.5890158434735834, P15=-2.1320194600940265, P16=-3.5931628444206978, P17=-3.669199150487629, P18=-3.6881115088031895, P19=-1.8891638996752393, P20=-8.177147383447704, P21=-5.274817879346645, P22=-4.2908444006967885, P23=-3.4909035510900015, P24=-5.837399406557846, P25=-1.999917151406511, P26=-4.229245567310234, P27=-1.0169663512327383, P28=-0.8241057247988943, P29=-1.1892387691662494, P30=-0.07905651324026483, P31=1.7178895130122005, P32=0.2552112650166197, P33=4.210373730463306, P34=4.490958441220601, P35=5.584675363984848, P36=1.7971572730653094, P37=2.96730176775449, P38=0.7522328009785141, P39=1.0519904316203739, P40=0.7790521132791342, P41=0.6862852358816892, P42=0.6687179904526288, P43=0.6482550206491529, P44=0.6046148830934017, P45=0.6121421326388031, P46=0.6042831761122734, P47=0.6481462908630896, P48=0.6649975301673046, P49=0.6996676990681067, P50=0.7674850833288681, P51=0.8309321636786059, P52=0.0017959467055169273, P53=0.0012522395397106714, P54=0.0003542260678711367, P55=0.0002821744512671791, P56=0.0016852989799537546, P57=0.018648901854338585, P58=0.04362112925114572, P59=0.0663283462550325, P60=0.038866616691845664, P61=0.0326305561237326, P62=0.014343123809714847, P63=0.0061360549642415834, P64=0.009706534156828902, P65=0.052627734211273265, P66=0.1833878492582155, P67=0.2466978381541458, P68=0.10963289502864905, P69=0.07290454922133326, P70=0.0907135639125722, P71=0.08800600734159429, P72=0.04330925170512839, P73=0.03290909764659466, P74=0.0677507382936415, P75=0.07758512117512625, P76=0.0697618559075904, P77=0.026655276252112114, P78=0.02929341776498571, P79=0.053923943302373226, P80=0.09563764971434786, P81=0.3181358272871094, P82=0.32510689765350076, P83=0.08675201305596934, P84=0.08323054385358421, P85=0.0674665769269473, P86=0.06441951898417149, P87=0.065448837530635, P88=0.06486894660259716, P89=0.02311882622314018, P90=0.013187642669491526, P91=0.009220234258866505, P92=0.0075535239450789495, P93=0.005807928174091304, P94=0.004248599777532425, P95=0.00425048542467269, P96=0.0029165518964938317, P97=0.0020564010924586464, P98=0.0074266187451547125, P99=0.01120132448345018, P100=0.0018277565441563374, P101=0.0012560650815714466, P102=0.0011253437486428133, P103=0.0013163252127656592, P104=0.001537081656459748, P105=0.0027269576485367424, P106=0.0022530759845993125, P107=0.002966173786297874, P108=0.0019369714499356387, P109=0.0016103232610018838, P110=0.0023007406905031563, P111=0.0019222759144047594, P112=0.00211318121690572, P113=0.0012827162174765543, P114=0.0005596484104014766, P115=0.00045969490452326425, P116=0.000491245230736883, P117=0.0008179458563780165, P118=0.0019970498902054435, P119=0.001352590219110093, P120=0.0016922413570650474, P121=0.0034837402565442224, P122=0.002455217154469695, P123=0.004286898534679371, P124=0.0069791322864971505, P125=0.00119801523186193, P126=0.001766512711033661, P127=0.0008050558379815128, P128=0.00035945103595746606, P129=0.00038295984516723344, P130=0.00044363190641481137, P131=0.00043976537484510244, P132=0.0004968940161414042, P133=0.0015850781079553937, P134=0.0016632807952661267, P135=0.0026424673456672157, P136=0.002816268279012726, P137=0.0011132095309458142, P138=0.0007982351635384403, P139=0.0005487070491520406, P140=0.0006364880775232072, P141=0.0020364134281764417, P142=0.002061837434225781, P143=0.001786066175221279, P144=0.0005404220848835123, P145=0.0005766126012130534, P146=0.0007008054970103425, P147=0.0004442802617277542, P148=0.0002681484317894134, P149=0.0002985123941532095, P150=0.00026317316020414694, P151=0.00038328609481946537, P152=0.00045588516815623774, P153=0.0007325651612661229, P154=0.000782863247920882, P155=0.0009402901735534762, P156=0.0013127760909936168, P157=0.00117252776488818, P158=0.0016130621959983868, P159=0.0024413370106967504, P160=0.0017800963406069494, P161=0.0019278731162800226, P162=0.0013393247592283003, P163=0.0006462553494961486, P164=0.0005929234630304786, P165=0.0004650923720225577, P166=0.0002794249310033818, P167=0.00024175892424394758, P168=0.00026290415443557045, P169=0.000283778868875924, P170=0.00020420598262594353, P171=0.00014104070991107629, P172=0.00012533783607159875, P173=8.580929385400961e-05, P174=7.558251028529783e-05, P175=6.263654480746909e-05, P176=7.018707888041598e-05, P177=7.681858440384404e-05, P178=5.61203785021102e-05, P179=9.514127350217828e-06),Row( P0=-600.7855959226699, P1=71.17761221992195, P2=-2.6760542380493724, P3=13.933074542767232, P4=-4.2139422815086975, P5=-8.559273641560228, P6=6.528799938895271, P7=1.1524201290978622, P8=5.357086169788648, P9=0.8602401989341275, P10=-9.797193023634414, P11=0.42193708489411486, P12=-2.181182950678162, P13=-5.542446320283313, P14=-1.1491948364606785, P15=0.2063709551740043, P16=-3.4110394298522775, P17=0.657382261754703, P18=-0.21970405726780146, P19=-0.6137607148979634, P20=-5.792210470478331, P21=-1.2325644289885709, P22=-3.7127368244511842, P23=-0.9876226492876973, P24=-3.6789405410359146, P25=-1.0298137091177186, P26=-5.3172802210777865, P27=0.45346322545306195, P28=-2.140794233434301, P29=-1.3752027172818675, P30=-1.431912789490609, P31=-1.8487397090900455, P32=-1.598856615480585, P33=0.013470236880749057, P34=0.1860605825850359, P35=0.032098687039346194, P36=-0.23871332248160693, P37=0.14589038028244425, P38=-1.387506367489513, P39=0.6148739621764754, P40=0.6770867396721897, P41=0.6712044834823245, P42=0.698630350641322, P43=0.6724824330387112, P44=0.6566114089183874, P45=0.7177769159673403, P46=0.7897626342446932, P47=0.8100303348426535, P48=0.7485240511034882, P49=0.7321958642774936, P50=0.7125226435162738, P51=0.7017437170888876, P52=3.244321268139065e-05, P53=0.0001158032984341475, P54=7.213233427142272e-05, P55=0.0001369418291597886, P56=0.006744875825506699, P57=0.04366246602599345, P58=0.0296166394675254, P59=0.02490458499371296, P60=0.012519945714359791, P61=0.005340838963784094, P62=0.016258575061221973, P63=0.03964186855951484, P64=0.020249162900914854, P65=0.01993481060602771, P66=0.06259234936118262, P67=0.06896457711412105, P68=0.036077139976987076, P69=0.0390170299478382, P70=0.025838370546968956, P71=0.009605489634100866, P72=0.006843112968882664, P73=0.01438639112894425, P74=0.03809666440191097, P75=0.03951683332230269, P76=0.04202957854263837, P77=0.03875377386413464, P78=0.02855748106333462, P79=0.022149393256727115, P80=0.03257695119354534, P81=0.07088134202652935, P82=0.09332603934021769, P83=0.04101806671320356, P84=0.030882468737372548, P85=0.02297422228430032, P86=0.023885518998480102, P87=0.030163230152152207, P88=0.04035853495941321, P89=0.048357769378225904, P90=0.024941090067107312, P91=0.023315645011440064, P92=0.08072974081495822, P93=0.06288243783563542, P94=0.05124955260787464, P95=0.049138981917809244, P96=0.06122894307086056, P97=0.026107931614937824, P98=0.015761978796657358, P99=0.012177314968867373, P100=0.00414731326786819, P101=0.002810043946936914, P102=0.0020407407366700107, P103=0.0034627293233658045, P104=0.0033972699763705366, P105=0.00392459834799654, P106=0.00516911405891743, P107=0.009972843882212456, P108=0.008936002905289288, P109=0.012106946125732324, P110=0.014289245683995625, P111=0.017619358166507063, P112=0.007735330571390784, P113=0.010658734334600349, P114=0.010028756631617921, P115=0.0038324306431944608, P116=0.009387058459667483, P117=0.0071084432650287705, P118=0.005035971633439164, P119=0.00565139745443973, P120=0.009428445423533856, P121=0.005878139377477053, P122=0.005930352822758958, P123=0.004355232323425559, P124=0.0035739458503807843, P125=0.0035863100094153323, P126=0.0044476409611957125, P127=0.0027574601528938883, P128=0.0010677437209938002, P129=0.0011485263542605304, P130=0.0007983517600845284, P131=0.0006167170664297336, P132=0.0008604377528052588, P133=0.0017991919267373484, P134=0.001948992235309379, P135=0.006202299056275843, P136=0.006128635707216794, P137=0.0074841200506166545, P138=0.005872637943772867, P139=0.002531764139370345, P140=0.0020492179280093554, P141=0.0021232842071125233, P142=0.0015861189319727968, P143=0.0016629573878000161, P144=0.0012603890583299236, P145=0.0012191425817168903, P146=0.0011783215239932934, P147=0.0008720680458511175, P148=0.0003898239693257347, P149=0.0004562783256400483, P150=0.0005421248609789802, P151=0.0007182006147061673, P152=0.0007455637278314693, P153=0.0010859474067737814, P154=0.0013095873144801803, P155=0.0019041465348849144, P156=0.002292153004532356, P157=0.0021300516779220085, P158=0.0017902830552223674, P159=0.001736369823264832, P160=0.0017793733683124963, P161=0.002417526461107204, P162=0.0017046639963503466, P163=0.0023383069843634792, P164=0.001899087312806855, P165=0.0012684650898861843, P166=0.0008938080085609597, P167=0.0007184965358930838, P168=0.0006378556818430061, P169=0.0008555950793793104, P170=0.0009306909934337076, P171=0.0007856410075732502, P172=0.0004889665284221848, P173=0.00040085787082872994, P174=0.00024422513412958335, P175=0.00015314637621689247, P176=0.0001089507159122567, P177=7.258600559985834e-05, P178=3.15697162450362e-05, P179=3.5568579010382264e-06),Row( P0=-615.9938305748979, P1=58.550303079523246, P2=-7.225581713479439, P3=3.496815546784894, P4=-9.990167782967436, P5=-11.6859835940804, P6=-6.4264363397226525, P7=-9.106291764073447, P8=-0.7113298105132745, P9=-6.074459741802201, P10=-10.643049403747918, P11=-4.025159363256746, P12=-7.657687480230494, P13=-7.424544771542958, P14=-1.2059806394821135, P15=-1.4308818874551286, P16=-3.575990163342651, P17=-3.109266200259895, P18=-2.8792061228220187, P19=-0.983857802876654, P20=-5.973179669622436, P21=-3.365438217489327, P22=-2.202408467421877, P23=-2.816231123161182, P24=-4.890317041425949, P25=-2.7716610717890067, P26=-5.094899481374879, P27=-1.962830098524031, P28=-2.7954118532224537, P29=-1.7229942867353747, P30=-0.25140328520049077, P31=0.6263017980401643, P32=0.6845935792274871, P33=4.745524807140858, P34=5.073488430906764, P35=5.447401554465938, P36=3.099269496690505, P37=3.32761157757883, P38=0.5634414927738872, P39=0.8673691116771353, P40=0.7598835554414284, P41=0.6877799287943251, P42=0.6744357756827344, P43=0.6592660932917922, P44=0.6180310507452114, P45=0.6313681788387392, P46=0.6762583444256182, P47=0.683597512204175, P48=0.6670026837840534, P49=0.7264137861872488, P50=0.7782849330116997, P51=0.7851322424334299, P52=2.825902149178955e-06, P53=9.644866404669474e-06, P54=1.0672302257974479e-05, P55=2.1712548301784355e-05, P56=0.0009527116180890004, P57=0.012145654690043644, P58=0.03889998094878016, P59=0.06406043953742202, P60=0.029410651730685567, P61=0.02647449798349444, P62=0.04366194187738431, P63=0.023960361338378557, P64=0.010423562726890214, P65=0.06582716869776664, P66=0.22540449899455403, P67=0.28997811873374907, P68=0.11124226542894214, P69=0.06396512627229513, P70=0.045298022775754655, P71=0.02698598627727803, P72=0.05821800560756789, P73=0.10371399193201745, P74=0.126896419245373, P75=0.10638987923380737, P76=0.05625783467424033, P77=0.03546732473786728, P78=0.04776394438337303, P79=0.043113868839764306, P80=0.04965435568275733, P81=0.12688358200050706, P82=0.1988939587154242, P83=0.11215997658511055, P84=0.07242783582538558, P85=0.035404620301336814, P86=0.036010845096310924, P87=0.07255377381322306, P88=0.23508332205775248, P89=0.2594162529164654, P90=0.09855217997077424, P91=0.02997118871541586, P92=0.017735450604794045, P93=0.06782952972537083, P94=0.15984430869177785, P95=0.06747327753780927, P96=0.010624043657892834, P97=0.0062194259670903945, P98=0.025422860624772715, P99=0.035833893705567395, P100=0.00694470542306295, P101=0.001728227823894258, P102=0.006849270000974408, P103=0.008077027477938082, P104=0.005583535190570744, P105=0.0036568431945439572, P106=0.004960413863235784, P107=0.00805909758788289, P108=0.0050282557843357775, P109=0.006240465202845519, P110=0.01132557216204344, P111=0.01718400324777718, P112=0.02378306240286501, P113=0.012696093948543152, P114=0.0013436684018896003, P115=0.001876378835063111, P116=0.008890593878625746, P117=0.0023038740479799525, P118=0.010759980071752502, P119=0.013155516518476497, P120=0.0027933056619411784, P121=0.004585380011433522, P122=0.0013849647954120511, P123=0.0019137434624308864, P124=0.0008846737531037996, P125=0.0005484113996806682, P126=0.0003680131661889096, P127=0.0003588021266696392, P128=0.00022744261044870462, P129=0.0001795522156014681, P130=0.00018194613636503883, P131=0.0001418009923157774, P132=0.0001565586396784053, P133=0.0004140742316540016, P134=0.00046420276167345243, P135=0.0012216708255838435, P136=0.0009263471824642073, P137=0.0003967701098677297, P138=0.00035911339649273203, P139=0.0003571839926233816, P140=0.0004296152531749352, P141=0.00044474417483695054, P142=0.00039743188325579745, P143=0.0006463472991042889, P144=0.000559712558952763, P145=0.0007118045630073732, P146=0.0007282714578846754, P147=0.0006482647432395679, P148=0.0005411189574640453, P149=0.00044820659767588793, P150=0.0004512985758409681, P151=0.0007914756267936307, P152=0.0009075186186123445, P153=0.0010734388965899873, P154=0.001100179969505354, P155=0.001464999652992693, P156=0.0014512371411952467, P157=0.0009763727682813916, P158=0.0006245733961549373, P159=0.0006855341645377263, P160=0.0006551611221513942, P161=0.0009774651215337688, P162=0.0008525276147864671, P163=0.0005523494116452039, P164=0.00040311836802405014, P165=0.0004691880645715116, P166=0.00033274976508113974, P167=0.00023704501848588092, P168=0.00029396925720384995, P169=0.0002608546452289803, P170=0.00024644365977698137, P171=0.00019625559804372137, P172=0.00015958610109715902, P173=0.0001695931579029331, P174=0.00014730929576671728, P175=0.0001289325864684631, P176=0.00013963767347306545, P177=0.00012417595538301033, P178=9.302600566657523e-05, P179=1.556101656656063e-05),Row( P0=-505.03103996783193, P1=50.908516749477855, P2=-20.266667735765747, P3=3.9805114350129704, P4=-16.194810984508976, P5=-20.333789820334967, P6=-8.558780894379305, P7=-8.621106410962573, P8=-4.932907175896648, P9=-4.3078496615149895, P10=-14.407316332969033, P11=-4.705959406186024, P12=-8.593338780905453, P13=-10.464558059838238, P14=-2.6968250564649408, P15=-3.9666518351777165, P16=-5.629624410980238, P17=-3.5819561514190976, P18=-5.431471828869442, P19=-1.1070598315582105, P20=-8.0118299070801, P21=-3.083648885535192, P22=-1.4750549387698992, P23=-0.5310022879989399, P24=0.6340356656010228, P25=2.3349051966335423, P26=2.0534700213925765, P27=3.6700886030853743, P28=4.709656947293617, P29=4.515393308214492, P30=2.9343979659568737, P31=2.170894657494032, P32=-1.1498258292035302, P33=0.3967866273449397, P34=-0.157241079514687, P35=0.8973837679468795, P36=-0.74744991379043, P37=1.9244983388707142, P38=-0.5645130852332477, P39=-0.7525534454840592, P40=0.728661601339435, P41=0.7206486736733357, P42=0.7102388596048849, P43=0.6958734122773034, P44=0.6838004706547751, P45=0.6953151467957641, P46=0.6567728639312794, P47=0.6378228616287892, P48=0.6639148533005521, P49=0.7297322972869443, P50=0.7439235605140845, P51=0.7383876585693997, P52=7.333649468322076e-06, P53=2.236932985095083e-05, P54=3.762719960847983e-05, P55=7.329336590891794e-05, P56=0.0002597040934827511, P57=0.0022401844552019314, P58=0.014652892497653542, P59=0.07057316687801676, P60=0.23654661540866054, P61=0.37354341484886033, P62=0.5272357720366654, P63=0.29740105258024496, P64=0.06202479035991937, P65=0.032512712432372054, P66=0.03571456173486354, P67=0.04830078462387966, P68=0.12230606747383166, P69=0.39625658265604946, P70=0.5048274948095777, P71=0.35285332808472064, P72=0.2351388778877093, P73=0.43709409951082395, P74=0.4862228902178631, P75=0.3837757141205922, P76=0.38289594637292024, P77=0.7031987712721927, P78=1.1459308479546302, P79=0.9861355269802856, P80=0.42683459035644883, P81=0.3033433199763848, P82=0.42938345666489197, P83=0.3847365811182002, P84=0.725337512039741, P85=0.8031528694979952, P86=0.4865230405287023, P87=0.5249399417521615, P88=0.3917577716664523, P89=0.5101901194330857, P90=0.4928286853190828, P91=0.3386331362850071, P92=0.10091808962907094, P93=0.41385399621668284, P94=0.5689662736447794, P95=0.17643774507632265, P96=0.2848097569214982, P97=0.1233599732836237, P98=0.03751653543323427, P99=0.05045286637888924, P100=0.05949198954043231, P101=0.04499141759496944, P102=0.015725065098736665, P103=0.022413236702903787, P104=0.0241248668472391, P105=0.02122693090338855, P106=0.027047834162106692, P107=0.046353128442199826, P108=0.03952729510930194, P109=0.02106420874677766, P110=0.03071975927564512, P111=0.030657880029884902, P112=0.06794355605168342, P113=0.054937678476137834, P114=0.021721067531776703, P115=0.04674722541990036, P116=0.05246484230449367, P117=0.03378362485437766, P118=0.08003477598675049, P119=0.05936071024729059, P120=0.0320748357805218, P121=0.10334527926519373, P122=0.029786097231035123, P123=0.043510406740098116, P124=0.035983305061319394, P125=0.009815943292324693, P126=0.009031210775727742, P127=0.00937982829338804, P128=0.005683109162625543, P129=0.006882247487411944, P130=0.005625448657593545, P131=0.008094160457607448, P132=0.010044767679742682, P133=0.045693304952766134, P134=0.029942332963809656, P135=0.035377752943974765, P136=0.02237778802284376, P137=0.011548420594900927, P138=0.01468624171044854, P139=0.02033014479965477, P140=0.024312105645055117, P141=0.03642597552472658, P142=0.014100677160817131, P143=0.012018167051336006, P144=0.006998085823790609, P145=0.0052374972320998366, P146=0.0040440830739343295, P147=0.0024317916039295045, P148=0.0014091312140856631, P149=0.0012869958173764866, P150=0.0016857040098095311, P151=0.0010308078472808645, P152=0.0008471880696944648, P153=0.0012800991890414976, P154=0.0020194471317289114, P155=0.0034202100603200523, P156=0.00483957336771529, P157=0.003974176305830223, P158=0.0015452639867579653, P159=0.0016124232734351387, P160=0.0021174230052625996, P161=0.0019086416672771211, P162=0.001617900802162799, P163=0.0016404766272765574, P164=0.0016068608627072042, P165=0.0012235611073625674, P166=0.000849405947530848, P167=0.0009408422141929644, P168=0.000606696602970468, P169=0.000524238035916365, P170=0.0005807811108026788, P171=0.00042899798157285075, P172=0.0003951379977534735, P173=0.00047216750406649526, P174=0.00030938285083256486, P175=0.00026374742771578303, P176=0.00025831002635834203, P177=0.00027959940619780966, P178=0.0001911844963679, P179=2.0027688387235222e-05),Row( P0=-573.1716452643083, P1=49.49536034801373, P2=-13.440800577781637, P3=5.835116665176549, P4=-14.49254430374152, P5=-17.777392075675905, P6=-0.4710906027867088, P7=-9.76552270452644, P8=-4.378000394428972, P9=-3.639683676462504, P10=-11.220084691468465, P11=0.030140553697126832, P12=-3.933203346406818, P13=-6.504232349937065, P14=-3.4020300132764705, P15=-3.3630547835252664, P16=-3.7664381859688083, P17=-2.2662738637282263, P18=-4.255774795894191, P19=-2.7263272117879387, P20=-5.803048257967555, P21=-1.8362536047191587, P22=-5.118625537933626, P23=-2.1118289864866213, P24=-2.4380303774327867, P25=0.011168041058918804, P26=-3.9852816271704907, P27=1.394428494877656, P28=-0.4034273324272452, P29=0.7861015017931836, P30=-0.9120877867367093, P31=-0.26456892574132784, P32=-1.432566634011878, P33=0.5119864226007917, P34=1.2491944756221154, P35=0.9099178656456528, P36=-0.220188078420302, P37=0.8661814296196917, P38=-0.04355294945223129, P39=1.0880248541830968, P40=0.7013733616589461, P41=0.6706505725834149, P42=0.7006190540941285, P43=0.6995316604433043, P44=0.6858367219765449, P45=0.7361714298592767, P46=0.7911660566360224, P47=0.7952840287269389, P48=0.780168385805695, P49=0.7754535764160071, P50=0.7531866639392916, P51=0.7381604548230539, P52=3.190185135033066e-06, P53=1.0340698379179497e-05, P54=1.7734431653633437e-05, P55=9.720718544905912e-05, P56=0.004103601940909547, P57=0.03207032901724471, P58=0.0232427112737701, P59=0.013767964472990446, P60=0.026873780547560944, P61=0.04779012122794291, P62=0.02969121196380695, P63=0.03116954978994466, P64=0.02536558125513675, P65=0.026491521281110153, P66=0.042642805472988675, P67=0.049871812874323906, P68=0.04600284736512509, P69=0.07696883766290079, P70=0.12944531263579476, P71=0.10475752747952885, P72=0.06146032860775788, P73=0.04345395705802767, P74=0.042931093738891736, P75=0.06887908338902139, P76=0.07085673624065122, P77=0.2138019378827107, P78=0.3246881806981411, P79=0.2252263784304693, P80=0.14147975226374856, P81=0.08219937573439544, P82=0.06041981021896316, P83=0.035617766554831384, P84=0.028221895412922145, P85=0.022919426399105384, P86=0.02615111356061189, P87=0.04867181359931097, P88=0.06416485598321188, P89=0.044329547469768395, P90=0.026524686278540717, P91=0.01754657183161446, P92=0.06508530451347368, P93=0.07135675610137628, P94=0.12230577873643367, P95=0.09638997760749739, P96=0.06826942752130953, P97=0.049034274089555435, P98=0.06550032476398208, P99=0.0542091892537685, P100=0.012588944286429843, P101=0.008823356384686815, P102=0.0063701758099288415, P103=0.004749495474428079, P104=0.0040049298300176995, P105=0.004222149448441463, P106=0.002842758440513102, P107=0.003282927030160545, P108=0.0036146306693362364, P109=0.002856649016155315, P110=0.011641619515441055, P111=0.01792621389204049, P112=0.015229877725143153, P113=0.014671933423955279, P114=0.008513746511507266, P115=0.013916692302241351, P116=0.030877603187780125, P117=0.03273398539453041, P118=0.017826065100948904, P119=0.020456200025019883, P120=0.015338920302862493, P121=0.00576851148483525, P122=0.004504809550186401, P123=0.003909015933103941, P124=0.0038991654076015957, P125=0.005971566798643013, P126=0.007613261968557228, P127=0.006935994236627823, P128=0.002796286968898533, P129=0.00249392779730249, P130=0.0023512520097019552, P131=0.00199269614767456, P132=0.002869722854405209, P133=0.004347996703198967, P134=0.0045860492386901805, P135=0.013657761764517228, P136=0.015250894285151815, P137=0.016344893317660442, P138=0.01498453776427516, P139=0.006430090659368101, P140=0.003381415436129029, P141=0.0039048195298727294, P142=0.003603856560972577, P143=0.005224493903550137, P144=0.00484677691746037, P145=0.004292447345345603, P146=0.00234846639305925, P147=0.0023803678836096386, P148=0.0010023551648140855, P149=0.0006383046126189263, P150=0.0005608347059434195, P151=0.0007109327343062326, P152=0.0005438484148327416, P153=0.0007733554294945267, P154=0.0011267324132988297, P155=0.0016457984282392717, P156=0.00149316798761086, P157=0.0018185078913534637, P158=0.001803030937742605, P159=0.0009528988802019296, P160=0.0009311198406621451, P161=0.0012924687811362954, P162=0.0012687606658759217, P163=0.0011308346801158565, P164=0.001015207495373976, P165=0.0008835577997740454, P166=0.00039264240685346743, P167=0.0002269223268073307, P168=0.0001658462459554758, P169=0.00016755726449957516, P170=0.00018397884111144303, P171=0.00015521199200687114, P172=0.00014019527540394365, P173=0.00014260870869573888, P174=0.00013192401320031238, P175=0.00012003043674307747, P176=8.140943323108427e-05, P177=5.53319870416003e-05, P178=4.049125018791547e-05, P179=4.715389462035887e-06),Row( P0=-499.4212404380864, P1=48.6128488780495, P2=-25.51506430690698, P3=4.292348019505679, P4=-18.709139769625008, P5=-18.684709452773056, P6=-4.604517737456466, P7=-11.186360330585, P8=-6.828254740019823, P9=-4.074875775411642, P10=-15.250895373526403, P11=-3.687980187156379, P12=-8.239455960368877, P13=-10.281173681587477, P14=-2.2662459561067623, P15=-6.965495535568308, P16=-2.045607641488963, P17=0.6594652154705556, P18=0.6488963282326753, P19=2.0361098300587686, P20=-1.3885491251772641, P21=-1.347232129835187, P22=-1.3188069616575673, P23=-1.9294919071441337, P24=-1.0060882099823312, P25=1.9494436635772627, P26=-2.4476792957794284, P27=0.6547587919599412, P28=1.580039017413024, P29=0.3790587816340678, P30=2.5857681978626874, P31=2.2934146862213267, P32=-0.08839985732684415, P33=1.8001847013466077, P34=0.40045642517455954, P35=0.14259160361020942, P36=-1.0447048345653587, P37=0.8679766355754838, P38=-0.2745071264323643, P39=0.09366912160668077, P40=0.7183208437133318, P41=0.6361860062609439, P42=0.6404251655949739, P43=0.6279106546521922, P44=0.6547557862886434, P45=0.6347453709860306, P46=0.6071597923097133, P47=0.6237923295655451, P48=0.6586412522587088, P49=0.6970199842929723, P50=0.7324071997567738, P51=0.7495031375902163, P52=1.1684287982735537e-05, P53=2.6348914655577636e-05, P54=5.2799203602605926e-05, P55=0.0001313589831260053, P56=0.0010046785294813016, P57=0.004430992268644947, P58=0.007945697078571913, P59=0.0339286089108287, P60=0.08784058181388768, P61=0.1246221094409669, P62=0.19805786332542002, P63=0.2569778364820246, P64=0.12231742150013078, P65=0.15660822771141295, P66=0.2522161734408019, P67=0.3500712170348573, P68=0.23398044866315926, P69=0.31634793216408774, P70=0.3939591570694116, P71=0.20110513067388147, P72=0.1180049938448156, P73=0.09825243176539655, P74=0.07871199744030642, P75=0.105680900683805, P76=0.18810330254845653, P77=0.5914864257129829, P78=1.197292988855387, P79=1.2696153846588565, P80=0.9247435969692899, P81=0.7142423977512566, P82=0.7358129562487566, P83=1.1794826384896102, P84=2.1286681217930874, P85=0.8264699744237392, P86=0.12287219030777347, P87=0.1300756969618256, P88=0.2999898230779338, P89=0.18139150597233228, P90=0.13835253335579384, P91=0.26758462311220016, P92=0.7049064347562695, P93=0.4214611925333868, P94=0.2930826382338989, P95=0.3598384769535424, P96=0.3031076049028485, P97=0.1036580686699276, P98=0.09796240916458006, P99=0.2437363910690097, P100=0.06383038439691321, P101=0.10594111615893315, P102=0.0469029098468215, P103=0.04003269265608316, P104=0.03352259031689986, P105=0.05503390479626624, P106=0.03693340945830222, P107=0.04801682862235181, P108=0.057033282973607755, P109=0.032003638552525665, P110=0.06174400328325813, P111=0.09079485883963595, P112=0.07520962718201214, P113=0.09310309840164217, P114=0.09301861218492946, P115=0.028041267290589103, P116=0.009992858561924023, P117=0.02123398947952452, P118=0.043691021294022754, P119=0.025054240723753958, P120=0.017544479239558715, P121=0.03075013061813056, P122=0.03508422220388007, P123=0.021063653911337778, P124=0.011423223958642191, P125=0.006704990608858321, P126=0.012584413354950873, P127=0.015497354073674951, P128=0.011368868663176173, P129=0.006123051726944471, P130=0.004661136416982441, P131=0.002371437047039665, P132=0.004381152877662505, P133=0.011196957924161479, P134=0.006811078901182604, P135=0.013774333972852755, P136=0.019456806166304626, P137=0.015202525850212539, P138=0.010755419154375548, P139=0.013566582026014077, P140=0.013906632941197688, P141=0.022184947348236634, P142=0.019764009777440732, P143=0.015791888442521138, P144=0.018100959550085793, P145=0.015218280351431447, P146=0.007652468069350594, P147=0.003520840160075501, P148=0.0012524210326955033, P149=0.0002803663500788754, P150=0.00026654045934871955, P151=0.00034636227163500075, P152=0.0004926835770554869, P153=0.0009750439174213023, P154=0.0006200464744938115, P155=0.000754001346412305, P156=0.0010020786869219076, P157=0.0007826949904261025, P158=0.00043198627947523384, P159=0.0007570678244868983, P160=0.000651998851807125, P161=0.0005551146974985734, P162=0.0008474595989119078, P163=0.0008106817844479303, P164=0.0004344553482326088, P165=0.00030562076148578355, P166=0.00019902657012299738, P167=0.00024169746960956242, P168=0.0002781551010995966, P169=0.00022458749518896906, P170=0.0002506453114167219, P171=0.00019863094338185694, P172=0.00013001343738006836, P173=0.0001354971516418029, P174=0.0001239265374279459, P175=8.235554863201694e-05, P176=0.00010830869037303286, P177=0.00012761626392844344, P178=7.041931176356035e-05, P179=8.042445014471466e-06),Row( P0=-594.3709041074495, P1=54.80914523867533, P2=-9.939239370249767, P3=8.898465411888525, P4=-10.313146623165663, P5=-11.996956064025214, P6=2.7536503829492838, P7=-5.295803193569053, P8=-0.5624342624702471, P9=-3.2211657624497763, P10=-11.440811330245358, P11=-0.9435765263474977, P12=-5.2522281749706, P13=-6.929752073516119, P14=-2.190754767099196, P15=-2.5179399382279546, P16=-4.270531638902042, P17=-2.061131693155362, P18=-2.177219850170422, P19=-0.8723181593444944, P20=-6.921256085847444, P21=-1.946817048204168, P22=-4.417120424227579, P23=-3.0877961757506545, P24=-4.517798234757206, P25=-3.1119578316961034, P26=-7.730926041830719, P27=-1.4292823660371738, P28=-4.046884443616092, P29=-1.9941111934369138, P30=-2.06243915802576, P31=-2.3881990716654764, P32=-2.6400489606384707, P33=-1.451374174861632, P34=-1.808161702804266, P35=-0.6302042987052654, P36=-0.6364607062402865, P37=1.0274558729879026, P38=0.04912434956164849, P39=1.2873748761466184, P40=0.6872412710401851, P41=0.6691168761733283, P42=0.6460962679474651, P43=0.6473715687161897, P44=0.6701686140144958, P45=0.7303610007541649, P46=0.7866125681498174, P47=0.8024490516180296, P48=0.7564669969012614, P49=0.7741979590733912, P50=0.7587291124180401, P51=0.6954236218340705, P52=2.0605217497401412e-06, P53=7.191432318102556e-06, P54=3.094984039144648e-05, P55=0.00013173096115078796, P56=0.0037971998540614065, P57=0.020746933808244182, P58=0.014746192724473398, P59=0.01423548292189962, P60=0.012273178378780083, P61=0.006418593705874644, P62=0.01145070953578743, P63=0.01774130188281399, P64=0.010580620551258295, P65=0.019107872269003044, P66=0.04306843561328679, P67=0.06454088952758016, P68=0.05148367665062192, P69=0.0421049008972508, P70=0.03628276914869086, P71=0.017559185552167702, P72=0.010327213091892742, P73=0.01476606993145759, P74=0.036326402098538574, P75=0.05055485942542448, P76=0.05735754563550757, P77=0.08502520799687141, P78=0.07564888547097907, P79=0.06555309017001626, P80=0.04640228765573014, P81=0.025014962518615194, P82=0.02180430264133253, P83=0.018926859206930553, P84=0.022556993303924352, P85=0.028292943461696377, P86=0.04173386087817739, P87=0.04210934350276827, P88=0.03048673757348445, P89=0.021854011574546965, P90=0.024485546740933423, P91=0.03234431103498198, P92=0.03478100301352115, P93=0.034967606979512615, P94=0.03865108773185136, P95=0.07132931877051557, P96=0.06598684997557452, P97=0.03271810552240044, P98=0.02653019290010473, P99=0.02869230949856081, P100=0.008392114588849073, P101=0.006233260510918036, P102=0.0035584769818054356, P103=0.0030933119945207774, P104=0.004638020021899401, P105=0.008323661832955524, P106=0.012040080480253882, P107=0.014014143314818678, P108=0.009780409711866602, P109=0.010015285050256329, P110=0.014786180259811483, P111=0.015241977180741125, P112=0.009484743997437274, P113=0.005165315056597938, P114=0.0034208331433447, P115=0.0035114139313634704, P116=0.005142245901785194, P117=0.008471632875354454, P118=0.008427030917812299, P119=0.005063035770910998, P120=0.006461191157427169, P121=0.010855516125018932, P122=0.011697939826877484, P123=0.011253229478425884, P124=0.006707438282526621, P125=0.004431574942361969, P126=0.007292370197483066, P127=0.008680188972766702, P128=0.002932724202289553, P129=0.002443520303133377, P130=0.0019243145522925224, P131=0.0015338070412328996, P132=0.0024507504873763123, P133=0.006441351116539192, P134=0.0052979904047827766, P135=0.008136215306156575, P136=0.007814090921206364, P137=0.010421965660142186, P138=0.00875207776722696, P139=0.004167147616642133, P140=0.0025680296116508444, P141=0.002823924201267723, P142=0.0026781796120890766, P143=0.0029691901489318916, P144=0.0019584552759124886, P145=0.0024212490086043766, P146=0.00171334550209894, P147=0.002115122692602363, P148=0.001366575274103027, P149=0.0011643728121718338, P150=0.001469472068904608, P151=0.0016315817082301026, P152=0.0014855172931951995, P153=0.0017150539696560267, P154=0.00244565710220046, P155=0.0023845087165838604, P156=0.003036008623033011, P157=0.002822274061806369, P158=0.0030272791111917597, P159=0.0029456628830366975, P160=0.003613161033963772, P161=0.006020735218485562, P162=0.004799276329063803, P163=0.004302409625076995, P164=0.0049012936124560496, P165=0.004705300459588695, P166=0.002693102019946422, P167=0.0015509467431976756, P168=0.0008808801041061899, P169=0.0005663393943394887, P170=0.0004093969357222085, P171=0.0002486318793458123, P172=0.00014606627909249638, P173=0.0001451934319209156, P174=0.00013582806425923072, P175=0.0002443876986990824, P176=0.0004135826367213365, P177=0.0004045743425588638, P178=0.00017514341602045325, P179=1.6557780018307575e-05),Row( P0=-621.6178567768144, P1=77.97032560261712, P2=-7.5121164729962535, P3=4.1185013377069355, P4=-8.777091418389475, P5=-12.922096967842766, P6=1.5861115415977924, P7=-3.6738650058636164, P8=2.18909182203449, P9=-2.7943532870339007, P10=-8.317621435918952, P11=-3.123336843793953, P12=-4.988887128731255, P13=-6.81477263619694, P14=-2.184451726565297, P15=-3.013803523142832, P16=-5.815540965106958, P17=-0.11247024236766576, P18=-3.396378120690725, P19=-4.859804189176653, P20=-7.9421718064693465, P21=-3.8975119909013816, P22=-5.1448125730595295, P23=-4.1264205285313205, P24=-4.760475536291267, P25=-1.7411030743116802, P26=-5.769730424925251, P27=-1.4878494962162239, P28=-3.353324948570145, P29=-3.0868573257447776, P30=-3.3201274684405466, P31=-2.9671846427388666, P32=-5.341929810549758, P33=-2.4603103384227114, P34=-1.9467207489770944, P35=-0.4285273853847385, P36=-2.2293513168162384, P37=0.17382403045196756, P38=1.0468329175485778, P39=2.403817290722912, P40=0.7186233898032758, P41=0.629250638833014, P42=0.6269738068301748, P43=0.6808009016276926, P44=0.6636958433392375, P45=0.6722561321484514, P46=0.6783852013756727, P47=0.6953607936313925, P48=0.7292032114952783, P49=0.744135637906854, P50=0.7010381152586589, P51=0.7042606715891018, P52=2.7701046848807025e-06, P53=9.828529870603435e-06, P54=3.576715786882617e-05, P55=0.0002498479141448888, P56=0.009328929106740196, P57=0.07235745037534383, P58=0.05622472439860369, P59=0.017172754587391606, P60=0.003116228448911081, P61=0.0018181752551976503, P62=0.013260297395357624, P63=0.0290074698499153, P64=0.029573865061680107, P65=0.015364281448123867, P66=0.025876903208229454, P67=0.027685335243839888, P68=0.00762337781521491, P69=0.00846915114318449, P70=0.033095851244389775, P71=0.02884620906562708, P72=0.014068593366610975, P73=0.01004760135339649, P74=0.01536742151308739, P75=0.011726190694003361, P76=0.007321224706721502, P77=0.012762629057873983, P78=0.03446175242899325, P79=0.05695177182409627, P80=0.031227474041588075, P81=0.06395426749749428, P82=0.10726182893209156, P83=0.04236543491360605, P84=0.01738096610876876, P85=0.0064382976959998, P86=0.0058899436239915354, P87=0.012272066630489046, P88=0.013947067748266442, P89=0.009637286363755583, P90=0.007663304571563947, P91=0.010760876534801555, P92=0.09631447937778644, P93=0.040331066545266844, P94=0.014735422634782305, P95=0.013553969567134528, P96=0.041603415711969, P97=0.01950688899271019, P98=0.004329405336946817, P99=0.002491476480490442, P100=0.0024979491236274934, P101=0.002403618360814938, P102=0.0006511438037214146, P103=0.0015701524115790421, P104=0.0009143729078687009, P105=0.0007607404636593692, P106=0.0014171599446574524, P107=0.0009366542651562276, P108=0.0017165848845766016, P109=0.0028894248768143835, P110=0.0030758646456476857, P111=0.012830339785966871, P112=0.007887245976774414, P113=0.002053672481637557, P114=0.0011649506118783228, P115=0.0006986899460920888, P116=0.0005232479846174081, P117=0.0006852118233241117, P118=0.0004818782464480429, P119=0.00036921657417020456, P120=0.00048728121721990146, P121=0.00048426050709111094, P122=0.00046335654733466967, P123=0.0004237272818135697, P124=0.0001359987347428082, P125=0.00013600595439701047, P126=0.00031398581725275437, P127=0.000555984209357705, P128=0.000247392648644967, P129=5.374232210325184e-05, P130=4.2419186858819e-05, P131=2.962787610611189e-05, P132=3.683635252178488e-05, P133=0.00012069503883041302, P134=0.00010907018109626743, P135=0.00025834481425471306, P136=0.00043952683244809476, P137=0.00035113058848816196, P138=0.000191453845535334, P139=0.00013980834426405387, P140=0.0001258254492699288, P141=0.00020259784360749267, P142=0.00017267094623172986, P143=0.0001675890788851609, P144=7.594345984990409e-05, P145=6.76726144465073e-05, P146=9.429258745869039e-05, P147=6.237786309684547e-05, P148=1.9084654466418602e-05, P149=9.239876117847657e-06, P150=1.545366173871546e-05, P151=2.3926361166844046e-05, P152=5.88327578024188e-05, P153=4.198660795402457e-05, P154=2.688825410221071e-05, P155=1.582907648640043e-05, P156=2.2942465476002145e-05, P157=2.7320237839203708e-05, P158=2.429821834522745e-05, P159=3.1439363503542826e-05, P160=2.0539448874513894e-05, P161=3.0072078067273325e-05, P162=2.0194052802389587e-05, P163=1.0556301417325234e-05, P164=8.648431472643788e-06, P165=9.27910595770904e-06, P166=8.727448225656394e-06, P167=7.568346976875326e-06, P168=1.0834677885152115e-05, P169=7.106135337536363e-06, P170=6.71249726949464e-06, P171=6.722585704585064e-06, P172=5.672313753488283e-06, P173=4.875464300913717e-06, P174=6.5399144570204026e-06, P175=4.958393590074911e-06, P176=5.280393668311908e-06, P177=5.587768974512577e-06, P178=3.4625890679399156e-06, P179=3.3639684096165434e-07),Row( P0=-651.3298156509818, P1=83.42103298278433, P2=-0.09459162773646754, P3=6.608381438407952, P4=-6.164172570042056, P5=-8.905817359535721, P6=1.5006511091893788, P7=-0.9537953303799241, P8=3.897439738715745, P9=-2.979465814905774, P10=-7.483239631083023, P11=-3.3126555506585467, P12=-3.308156591238387, P13=-6.239477335056746, P14=-2.5156388413335096, P15=-2.0662515271065574, P16=-4.985818606636207, P17=-0.6385255910270826, P18=-1.5670701995133363, P19=-3.331622636492215, P20=-6.604673444846027, P21=-2.961150679170359, P22=-3.7708687079092704, P23=-2.8544946784355005, P24=-3.7300674998368515, P25=-0.5435520515438164, P26=-6.388307899004439, P27=-1.2285273462183566, P28=-2.7395903516086975, P29=-2.867748139080728, P30=-2.784589149988938, P31=-1.6448182889565415, P32=-3.3059839742038104, P33=-0.9572343330582225, P34=-0.8311344440992771, P35=0.917130745310793, P36=-1.886213370564433, P37=-0.3948282733202384, P38=-0.2275351300981361, P39=0.3661013812287395, P40=0.7214033179622232, P41=0.6135411745344994, P42=0.6098974642435863, P43=0.6502286170000914, P44=0.62567190930133, P45=0.698626877807818, P46=0.7212913313429643, P47=0.7535048880729939, P48=0.7453336975389858, P49=0.7223652286227883, P50=0.6995760284197715, P51=0.7494188783088274, P52=5.576987356234268e-06, P53=1.6909208086063764e-05, P54=2.9464833543826004e-05, P55=0.0002455681927305967, P56=0.010820485618212545, P57=0.08533464767110556, P58=0.05235747954461842, P59=0.019205817702612826, P60=0.0038831443235427827, P61=0.001918349692637104, P62=0.006957835893980108, P63=0.023556717543882208, P64=0.01980336579434166, P65=0.012127596042800854, P66=0.03925993943634387, P67=0.056587534955536756, P68=0.01731234776947627, P69=0.007109067183471155, P70=0.011618316382172244, P71=0.00470953634384005, P72=0.004897255215642175, P73=0.023870008231267694, P74=0.029500839644966127, P75=0.01960390445380167, P76=0.010966423883711643, P77=0.004544695751553026, P78=0.0036496164490157318, P79=0.003973995073980417, P80=0.008062694887332902, P81=0.015546004946419362, P82=0.02819998317170796, P83=0.022109277959639285, P84=0.006844711605533348, P85=0.001384814264377376, P86=0.0009342136953563824, P87=0.007362759310278857, P88=0.016777876302663502, P89=0.005162162519777509, P90=0.0021757664943078928, P91=0.003236368416358899, P92=0.07709356218710833, P93=0.06437370463698867, P94=0.005759417500433019, P95=0.004381904039988334, P96=0.02727413088178305, P97=0.02854167609743436, P98=0.0012095410462354466, P99=0.0011499720186878644, P100=0.0018216894588717382, P101=0.0008376988818789012, P102=0.00012048165464996344, P103=0.0007838470708187931, P104=0.0004668370384458595, P105=0.00021720601882197185, P106=0.0015218056268921129, P107=0.001158630053556958, P108=0.0003966201525895682, P109=0.003539552601439439, P110=0.0015953068389790953, P111=0.003938349997136459, P112=0.0061032081863118945, P113=0.0006135680771494469, P114=0.000982130404183261, P115=0.0001545263266649235, P116=0.00035342951879007475, P117=0.00017704969042002194, P118=0.0002527842966170018, P119=0.0001743842171013225, P120=0.00014081230345932183, P121=0.0005258213964653418, P122=9.622542926349832e-05, P123=0.0001081787969459995, P124=9.503902721773795e-05, P125=7.684362193281407e-05, P126=5.886612185450543e-05, P127=9.71663200908246e-05, P128=3.4989107500709384e-05, P129=2.5904752673167073e-05, P130=1.4507462216342791e-05, P131=1.5134700216383903e-05, P132=2.2748854177831778e-05, P133=5.642734677498574e-05, P134=3.621448312414954e-05, P135=8.413441350241419e-05, P136=0.00010838009231274957, P137=7.165983187036461e-05, P138=5.5364364682334095e-05, P139=5.832778487370581e-05, P140=6.123971959236718e-05, P141=5.122208235982702e-05, P142=3.9737001307723036e-05, P143=4.042850444042876e-05, P144=3.555944616853527e-05, P145=2.5948547840708118e-05, P146=4.3714806061545976e-05, P147=4.968097207063019e-05, P148=1.3871199025708964e-05, P149=7.654572584624723e-06, P150=6.7423027227392964e-06, P151=1.7104452222098006e-05, P152=2.0491934901662726e-05, P153=3.1848794009272086e-05, P154=0.00011144262594561978, P155=7.835977584439158e-05, P156=4.7082054665807296e-05, P157=2.134121136244728e-05, P158=3.8566897889248964e-05, P159=2.477112145823963e-05, P160=1.7635449192439412e-05, P161=2.591692402669489e-05, P162=2.1182040124878404e-05, P163=1.955886655937721e-05, P164=1.6392790819269283e-05, P165=1.0792567982447485e-05, P166=1.0094930862517325e-05, P167=9.813281281227782e-06, P168=2.103031418206102e-05, P169=1.2720678708516089e-05, P170=8.550080952166843e-06, P171=5.006739314717452e-06, P172=6.866219845004306e-06, P173=7.240067757654208e-06, P174=7.243264335009853e-06, P175=7.69916924905685e-06, P176=6.343916090799539e-06, P177=6.7931027404154235e-06, P178=4.565523502106142e-06, P179=3.9976660890912106e-07),Row( P0=-521.6567515980735, P1=44.2174365125705, P2=-22.572256320650077, P3=6.11224034702032, P4=-15.14682703976648, P5=-17.404481602033318, P6=-2.891111680385444, P7=-4.5069984985398355, P8=-3.230049362960817, P9=-2.2766460314967407, P10=-13.87564842363208, P11=-2.329954609363858, P12=-7.076354424654846, P13=-9.138288001412484, P14=0.22070161127559687, P15=-2.144728482008755, P16=-5.468861623201156, P17=-3.847008453347342, P18=-5.2204979722168865, P19=-1.4929507151949346, P20=-7.197853668618529, P21=-0.6109709843716029, P22=0.1121952439537574, P23=1.0622368811786904, P24=2.2706708495766073, P25=1.3389659164243204, P26=0.7400759061196936, P27=2.1012755417318885, P28=2.791088445657185, P29=4.019192238393581, P30=3.278979977449612, P31=1.399020652214483, P32=-1.2949715092268272, P33=-0.9808924818595973, P34=-0.21521038975260212, P35=2.70395395498615, P36=0.037712202284579485, P37=1.5377447208388046, P38=-0.32211374259354264, P39=-0.28512198313537407, P40=0.7477959211314718, P41=0.7055707663681049, P42=0.7215280726762545, P43=0.67417259482155, P44=0.6252537129449232, P45=0.6530075905479249, P46=0.6630777236313713, P47=0.6625986130526208, P48=0.6669034552079288, P49=0.6870259388805062, P50=0.7206943413656265, P51=0.7536202944436219, P52=2.8848617153650903e-05, P53=4.073469980724743e-05, P54=7.449565196815375e-05, P55=0.00013159135379918317, P56=0.0003343962148154495, P57=0.0021947248500523084, P58=0.012432573167468791, P59=0.05443328042250024, P60=0.17505583504133382, P61=0.3558209097367467, P62=0.5541409747650982, P63=0.3825789193269113, P64=0.07966918993243108, P65=0.017548934592576296, P66=0.04348188094173097, P67=0.07517804833558529, P68=0.15143723867182698, P69=0.37465826847388245, P70=0.6772139526439535, P71=0.45475391573199314, P72=0.4655785121252955, P73=0.5872756096178343, P74=0.3026169420683498, P75=0.09413811484083118, P76=0.08698815866229187, P77=0.12049931900321562, P78=0.26195923805619814, P79=0.5624493457377184, P80=0.7491623594451752, P81=0.5349155809513502, P82=0.45605954464557624, P83=0.4436577962627194, P84=0.3941216750204388, P85=0.31079110076760463, P86=0.39725188529479194, P87=0.6659905649654992, P88=0.8181264835317009, P89=0.3338044384963583, P90=0.11930297884593649, P91=0.13540121432757682, P92=0.5098116051276029, P93=0.4593203089074722, P94=0.6888967724586145, P95=0.5514945705220786, P96=0.5527372769046923, P97=0.2435070555649368, P98=0.11616427026780994, P99=0.0627072684268662, P100=0.013422111787618356, P101=0.022581233077012205, P102=0.04536980879908873, P103=0.04479475383117329, P104=0.03232201686027428, P105=0.018031064467152615, P106=0.019610850277433135, P107=0.03785342062700302, P108=0.06197004363793876, P109=0.08829846269958687, P110=0.06891522793789284, P111=0.06283791414607577, P112=0.062107118616339746, P113=0.057589679204726435, P114=0.02660660221712578, P115=0.036296022392278436, P116=0.06802928417499197, P117=0.02434102131028875, P118=0.01220894391952674, P119=0.0348114220718492, P120=0.042352271977647994, P121=0.018691401902478054, P122=0.020132160470497883, P123=0.013388207463636843, P124=0.007446826010366813, P125=0.012401819776305844, P126=0.011773989873643012, P127=0.008472255976477506, P128=0.006382318203969489, P129=0.007901663259553285, P130=0.00434072584799587, P131=0.004237234354323611, P132=0.005932032196883035, P133=0.00919948157868972, P134=0.016591835342513173, P135=0.027590551016877792, P136=0.022321582176484066, P137=0.01905230528531646, P138=0.019456604871472327, P139=0.01647895299250833, P140=0.021918003026379855, P141=0.024145005008243382, P142=0.026491787422925708, P143=0.010225831535948126, P144=0.0057703231705873706, P145=0.005207127635577121, P146=0.003986806399037669, P147=0.003025274708097736, P148=0.0005230595647647576, P149=0.00023761409944771277, P150=0.0001642417538714325, P151=0.00018130716675647366, P152=0.00025868416342181977, P153=0.0001982983509744079, P154=0.0003690783100054738, P155=0.0003971144989892697, P156=0.00023316946626782744, P157=0.00020345290731909235, P158=0.00041991055486057265, P159=0.000190763132980006, P160=0.000269407016064801, P161=0.00036899484430551584, P162=0.00027895979802311207, P163=0.0001922092708615317, P164=0.00016309784229693287, P165=0.0002226393622698432, P166=0.0001556493500753155, P167=0.00016028451462080055, P168=0.00016997607046657408, P169=0.00011355226686236416, P170=0.00013260427628696795, P171=0.0001673615731234471, P172=0.00014084200223599496, P173=9.384342046387596e-05, P174=8.053791584636768e-05, P175=7.375742352028e-05, P176=6.639480619610998e-05, P177=9.661591857693822e-05, P178=5.0481064849553875e-05, P179=4.877905194793914e-06),Row( P0=-611.7125082651435, P1=74.76998631756176, P2=-10.916507063884083, P3=9.758970792767787, P4=-13.060818579987025, P5=-12.154790032690753, P6=-1.7565864965931572, P7=-2.372147408947901, P8=2.511303211865706, P9=-5.532309011703939, P10=-9.14417112840662, P11=-1.8951354439474022, P12=-5.00645192130624, P13=-8.987868072276452, P14=-3.9063186110638606, P15=-3.989732039909996, P16=-5.7177553451116685, P17=0.4052964800490132, P18=-2.0514083510445276, P19=-3.2363694846107642, P20=-8.115302725126986, P21=-4.9995539872934645, P22=-5.422157507212408, P23=-4.996816067340357, P24=-5.797515445459985, P25=-1.9861306687541957, P26=-7.177228724884023, P27=-1.7679658594943337, P28=-2.8512154353782644, P29=-3.3016266564335757, P30=-2.9015771429009027, P31=-4.35695833622968, P32=-5.047641886945466, P33=-2.530239182008825, P34=-1.3766115951877884, P35=0.03644321464173067, P36=-1.6689846149516965, P37=1.531941687086608, P38=0.5908200552658709, P39=1.2961622418348944, P40=0.6067497767644242, P41=0.5525958644104381, P42=0.6297340632537554, P43=0.6079817293429913, P44=0.6358492129664256, P45=0.6966472328305567, P46=0.6929165217979971, P47=0.7440332840465363, P48=0.7499610361503131, P49=0.7658003814247502, P50=0.7594254762341633, P51=0.6909968917078502, P52=1.3795243718387745e-06, P53=5.328930432318535e-06, P54=3.6276300026867726e-05, P55=0.0002568179770172025, P56=0.008333092037233734, P57=0.08863999834184308, P58=0.06400573565209848, P59=0.012747674598914693, P60=0.002352733945852336, P61=0.001666890688627278, P62=0.005569435316875169, P63=0.016374369584772685, P64=0.017370255033148572, P65=0.03515910625431272, P66=0.05322946793453881, P67=0.015993252119375242, P68=0.0038711975895158687, P69=0.006871348274778666, P70=0.021654617977847082, P71=0.01231262583193038, P72=0.010335171232378992, P73=0.031798631610021444, P74=0.01962385652584075, P75=0.004908986039153419, P76=0.007076151348711011, P77=0.008000711861287133, P78=0.007289730170166759, P79=0.02048491509752388, P80=0.07429307402491084, P81=0.08683047890433895, P82=0.05054390046857388, P83=0.01616366529681875, P84=0.005179404829279228, P85=0.0016104310659101337, P86=0.0008790934216365729, P87=0.006112747182469507, P88=0.01049064261769416, P89=0.003419121944407802, P90=0.000996540451297898, P91=0.001287842121336704, P92=0.0506698967895319, P93=0.03392925714356652, P94=0.002077107080289473, P95=0.0016018758321822422, P96=0.018933709156069747, P97=0.015334269645111986, P98=0.0005342785991637999, P99=0.00099648096978396, P100=0.001221503237354589, P101=0.0004415929015768216, P102=0.00023974149073479695, P103=0.0006627633369666848, P104=0.0002664095760727149, P105=0.00016419496789411064, P106=0.000550917726840333, P107=0.0003088273628254695, P108=0.0001761055899797934, P109=0.0004925474366687165, P110=0.00022409875914188805, P111=0.004519994190621808, P112=0.0028793605066550253, P113=0.0005728894235031773, P114=0.0008748920190014923, P115=0.00013774129056613876, P116=0.00021897037643697385, P117=0.0001878537104388135, P118=0.00029992328355585063, P119=0.00022428952265185872, P120=0.00019802683532340592, P121=0.00024745129488314724, P122=0.00019352165277878658, P123=0.00020405955389465424, P124=0.00011954080601730593, P125=6.376896891265312e-05, P126=6.610917091750245e-05, P127=0.00010496232011400669, P128=3.9509328138522075e-05, P129=2.2365578635122364e-05, P130=2.3509463385876072e-05, P131=1.8673534469198112e-05, P132=1.2874742171691832e-05, P133=3.104724276001474e-05, P134=3.097090228163173e-05, P135=9.979867809360761e-05, P136=0.0001887844359585859, P137=9.327825671639642e-05, P138=6.507446617047046e-05, P139=7.99699176312565e-05, P140=5.8628677454070335e-05, P141=0.000108774736878436, P142=0.00013694371814641657, P143=7.500222639535571e-05, P144=6.542060891389758e-05, P145=6.331466647097406e-05, P146=9.73404695475568e-05, P147=6.936580184244694e-05, P148=2.3942309792811896e-05, P149=1.448133401295916e-05, P150=1.6746478380195244e-05, P151=1.3922506347478162e-05, P152=3.569290688269521e-05, P153=6.524541289638737e-05, P154=7.394768298389421e-05, P155=6.993311245537393e-05, P156=5.9848819850901124e-05, P157=4.731453636555829e-05, P158=6.225024457357165e-05, P159=6.581771636659183e-05, P160=0.00010377288227283977, P161=0.00011549454173159147, P162=8.902211315217897e-05, P163=4.469474320240822e-05, P164=1.9945946614149106e-05, P165=1.7022454356899232e-05, P166=1.5929093473058937e-05, P167=1.5216052339129346e-05, P168=1.8598693727722295e-05, P169=1.3802247597668799e-05, P170=1.1201865205030659e-05, P171=1.1475779727192819e-05, P172=1.0698050341236605e-05, P173=8.159309993387013e-06, P174=5.857459112967689e-06, P175=6.183577111543972e-06, P176=7.2522189576611844e-06, P177=7.385974967308365e-06, P178=5.8194273077153276e-06, P179=8.230810826632046e-07),Row( P0=-573.7226366404781, P1=62.70226267989615, P2=-19.597656241989878, P3=8.419093403396667, P4=-16.422480468391633, P5=-17.701935601180764, P6=-3.290632990106581, P7=-5.260911808429817, P8=-0.616590567761363, P9=-5.694348520964382, P10=-11.561823295820986, P11=-1.9928992050893004, P12=-7.83680056462124, P13=-6.318973102238872, P14=-0.41579642146851914, P15=-3.4271765913038523, P16=-4.6018026431661765, P17=-0.43729667867571465, P18=-2.007841756774975, P19=-3.011747412028471, P20=-7.1175990348301195, P21=-1.4981915055062334, P22=-1.5378227162380007, P23=-2.27482110507135, P24=-1.4337436115736064, P25=2.1342864924159466, P26=-2.6295807214655924, P27=0.44700668016876166, P28=0.3418502163727789, P29=-2.7160782189171875, P30=-1.5374942946527193, P31=-2.696288330489889, P32=-2.8371795605510117, P33=-0.3457480413808012, P34=2.611098560827733, P35=4.504561708672488, P36=3.718735615211551, P37=6.166533673115088, P38=2.8843755163229696, P39=2.1803353172937348, P40=0.6673291604206483, P41=0.6182087573473789, P42=0.6133279688788837, P43=0.6131936066959488, P44=0.6260481797690136, P45=0.681275921154309, P46=0.6285321699641938, P47=0.5945404021844585, P48=0.6109575770956501, P49=0.6817346369225543, P50=0.7938482684813549, P51=0.7330329756276047, P52=4.745108705412025e-06, P53=1.6815242526900737e-05, P54=2.635619788561684e-05, P55=0.00012341836201047312, P56=0.0018673617185025377, P57=0.01682129759562844, P58=0.03986940326470143, P59=0.03129409576686186, P60=0.027219001036027278, P61=0.06277901627425111, P62=0.08706520221332541, P63=0.03923860507902123, P64=0.015026667413951155, P65=0.23298686281675923, P66=0.2704045426109872, P67=0.055553210848654735, P68=0.023564431676316812, P69=0.028938587144570406, P70=0.04177829143907973, P71=0.048024404670945535, P72=0.10383535559868555, P73=0.24683379122591917, P74=0.1603131733199139, P75=0.026785999922679726, P76=0.02200719290862481, P77=0.020488115679752457, P78=0.0293554292364751, P79=0.09675443531235198, P80=0.3269329370413205, P81=0.2957744865705645, P82=0.14826970863496808, P83=0.04483528722205364, P84=0.054272000904807015, P85=0.11676565425013996, P86=0.12490437176831685, P87=0.18580110869817673, P88=0.19690732583996678, P89=0.07829594798633487, P90=0.1063738882928728, P91=0.11303124820841823, P92=0.17803439843177316, P93=0.25834385803515686, P94=0.195026020364721, P95=0.19467600665997914, P96=0.290719894502943, P97=0.15172545107673102, P98=0.044822977900757545, P99=0.01563885886565296, P100=0.009051136059030311, P101=0.007969566742547802, P102=0.005692766537209505, P103=0.006044274445347158, P104=0.007771365208825307, P105=0.009290988488658555, P106=0.005472256532964751, P107=0.010665024391175271, P108=0.015870775263615196, P109=0.022094079817827163, P110=0.03261186250616126, P111=0.035766564030600856, P112=0.03314225536315518, P113=0.03768406833112057, P114=0.008556832553841692, P115=0.007296643097162686, P116=0.0171514242285481, P117=0.027219717580777063, P118=0.009672489806226366, P119=0.007587340671870888, P120=0.012322252042772491, P121=0.006046927668520826, P122=0.009469891640082264, P123=0.00495893005566494, P124=0.004529375286957622, P125=0.003114000309945924, P126=0.00485708270062759, P127=0.0023405890258431025, P128=0.0009527984731483044, P129=0.0004894345680464185, P130=0.0005275928979477235, P131=0.00044307806421984755, P132=0.0005140078707001653, P133=0.0018508073501543073, P134=0.00231945082345283, P135=0.004873561383314828, P136=0.005627657585899276, P137=0.003177719899385361, P138=0.0018457174068084708, P139=0.0020335844183563285, P140=0.0022901248895484784, P141=0.002936318875022925, P142=0.0027832826712213448, P143=0.002320716184566617, P144=0.0021997347164721156, P145=0.0008872278048638962, P146=0.00088583632716666, P147=0.0006212244017375056, P148=0.0004463584151846888, P149=0.00020508159377025155, P150=0.0004221662857197885, P151=0.0007290458269897117, P152=0.0009599984319837022, P153=0.0014346716229406379, P154=0.001609147971631655, P155=0.001205035483453708, P156=0.0012077532970286575, P157=0.001697959524323484, P158=0.0017589921166719813, P159=0.0013884035148065425, P160=0.001967309852280851, P161=0.0025141735123135897, P162=0.0027957373417921865, P163=0.001986311515723334, P164=0.0008477753424126615, P165=0.0005626720982990247, P166=0.00037076049386410985, P167=0.0002560232261632835, P168=0.00037265113149329715, P169=0.0003987085100569639, P170=0.0003495123538051526, P171=0.0002539931045920881, P172=0.00014757054622603787, P173=9.621336314866985e-05, P174=7.966213859574469e-05, P175=6.43825333537777e-05, P176=4.7810241580291766e-05, P177=5.9392800544961576e-05, P178=3.511389058671415e-05, P179=4.9972095944054315e-06),Row( P0=-680.989563235636, P1=54.45681058854423, P2=-10.247965404566907, P3=1.9212615467411123, P4=-7.491156617827846, P5=-12.191125461386589, P6=-4.493453567687503, P7=-5.441407012635622, P8=-3.85247312637929, P9=-3.8423141636489517, P10=-7.534268156084322, P11=-4.618941225685616, P12=-1.839474542031141, P13=-7.644636943244584, P14=-4.788781613404378, P15=-3.3987154484169997, P16=-4.168313985229369, P17=-2.0236375462610883, P18=-2.2255423354122095, P19=2.8940976441081294, P20=-3.6833468953518094, P21=1.658661386302577, P22=-0.4645634204635644, P23=0.7313251147903912, P24=0.47470361727677074, P25=2.2351997786104794, P26=1.7011636228912632, P27=3.0327996320674444, P28=2.9165280092171026, P29=4.043882971672729, P30=0.13912211331695382, P31=3.062084762159703, P32=0.6420312599800522, P33=2.1499941433607446, P34=3.80968504039462, P35=1.8372941583198503, P36=1.0619652597823166, P37=1.3474269438697963, P38=-0.5802603182119469, P39=0.8707963580659566, P40=0.734636078246346, P41=0.6927928553245177, P42=0.7165063622094257, P43=0.7218903940124081, P44=0.7178257420595656, P45=0.7119599268169051, P46=0.7032738659525102, P47=0.6845208703881263, P48=0.7001119072549078, P49=0.7213735679301021, P50=0.7319863288422842, P51=0.7404997541130826, P52=3.351166527781106e-06, P53=1.174689576665362e-05, P54=9.387616975544959e-06, P55=9.954365010293707e-06, P56=1.1104698592933089e-05, P57=0.0008024692104864349, P58=0.010920048626029874, P59=0.022799082541961488, P60=0.03315975891663153, P61=0.0410719258423593, P62=0.031192442050971873, P63=0.03156759718609448, P64=0.022085716215091342, P65=0.024075982378947904, P66=0.011764157175777985, P67=0.007417367903860293, P68=0.0054373234321149575, P69=0.009904432011363139, P70=0.018999449962921404, P71=0.01691607526872684, P72=0.009113863409958258, P73=0.007279982825498431, P74=0.009089571521718715, P75=0.00642162725181225, P76=0.0043497321313220195, P77=0.012170035272173314, P78=0.03513808019533625, P79=0.04851324541158591, P80=0.034315585130309995, P81=0.024919430578425397, P82=0.02053884742478499, P83=0.012970269331016724, P84=0.016269756708202375, P85=0.013424713041431459, P86=0.00626856005520176, P87=0.006036728372622682, P88=0.017273368687611423, P89=0.01632393058436934, P90=0.01261987010533139, P91=0.018716014487754, P92=0.01769622232931563, P93=0.009491270946618906, P94=0.009830992006065372, P95=0.04559633542205604, P96=0.03027828830155516, P97=0.018741446481242204, P98=0.01372721553732076, P99=0.003258772182483121, P100=0.0012029969211486864, P101=0.0008418634331984451, P102=0.0012558006564237572, P103=0.0015676919027693625, P104=0.003106571985038645, P105=0.004671945075550234, P106=0.005811396000163603, P107=0.002530561190939334, P108=0.001994263159255447, P109=0.005101111913237081, P110=0.012167813446695614, P111=0.010329796408309297, P112=0.0076688071767660425, P113=0.010326081281824133, P114=0.006812319171622572, P115=0.0025824135055344303, P116=0.001493125415108713, P117=0.0014507735130401265, P118=0.0007821555195559736, P119=0.0007124574709336809, P120=0.0003896975073220516, P121=0.00041769097832952635, P122=0.0004450009733624672, P123=0.00021111025263597687, P124=0.0006292103418457126, P125=0.0009651403205925101, P126=0.0002206901306049652, P127=0.00010706820018113093, P128=8.833450068286978e-05, P129=0.00010042051746519857, P130=9.139157809744369e-05, P131=0.0001760296060236345, P132=0.0003117592933719732, P133=0.0005051896318351211, P134=0.0011607271320528644, P135=0.0011334414657287353, P136=0.0006555821957221772, P137=0.0003984973235876328, P138=0.00038967171575005836, P139=0.00021776037294435612, P140=0.0002313840113818617, P141=0.00026460705752479654, P142=0.0003396999235855879, P143=0.00020756111266693778, P144=0.00010250184228719413, P145=5.136246727186107e-05, P146=3.550551656861574e-05, P147=2.76626494814657e-05, P148=2.5611410776308777e-05, P149=4.11081987253911e-05, P150=6.200351130669819e-05, P151=5.350281461297156e-05, P152=8.11069623185748e-05, P153=5.9324027357435e-05, P154=4.3693144165169226e-05, P155=5.886947287647446e-05, P156=6.876912249309196e-05, P157=6.808317564193327e-05, P158=0.00010638164659367239, P159=0.00013642274315824553, P160=7.743203189791553e-05, P161=4.22367842904108e-05, P162=3.4699482209591126e-05, P163=2.6349439230828715e-05, P164=2.5448958053775105e-05, P165=3.200184473870094e-05, P166=4.407630087677526e-05, P167=3.174302896666717e-05, P168=3.853351770573156e-05, P169=3.546228407464131e-05, P170=3.2328748425648305e-05, P171=3.0307201999966724e-05, P172=2.5075533323461433e-05, P173=3.108376703468255e-05, P174=2.281150881099697e-05, P175=3.35894564851485e-05, P176=2.7371524573326552e-05, P177=1.7313476872863224e-05, P178=4.901686146494818e-06, P179=6.195926194989794e-07),Row( P0=-576.1430580085316, P1=38.35011491000464, P2=-18.781669093167118, P3=-2.095011532147926, P4=-15.064012198735266, P5=-18.65613510747862, P6=-7.782363159030514, P7=-7.404881152904694, P8=-2.9851981042400624, P9=-6.45368673489497, P10=-9.390214979353434, P11=-7.277668816624408, P12=-5.915734141358192, P13=-8.848162306012924, P14=-5.617999061340657, P15=-6.470389941775473, P16=-4.4920093628933735, P17=0.7339157461728502, P18=-0.4488466975770473, P19=10.499844390256083, P20=5.990170430528425, P21=8.526763855783775, P22=4.810483708089218, P23=2.261511556225372, P24=-1.671592163463812, P25=0.7866306217907081, P26=0.6322170071852935, P27=1.3831483595783738, P28=-0.8952045697326172, P29=-1.076960686555624, P30=-4.724074074626222, P31=-0.4687010565323639, P32=-3.466244078292422, P33=-0.3952119901247802, P34=-4.943715026030608, P35=-3.9360558953227427, P36=-2.045156060183353, P37=1.0004722471482805, P38=2.0000826700325725, P39=2.727879018352619, P40=0.5454109484643244, P41=0.5668359549323965, P42=0.5801305474753004, P43=0.596562327300811, P44=0.6629890911408945, P45=0.6355358729635359, P46=0.658517987317108, P47=0.6788042702976536, P48=0.6765242841794844, P49=0.752391148693447, P50=0.6456022602947755, P51=0.552284865640825, P52=3.2569165386270925e-06, P53=8.04594452501223e-06, P54=1.837559311816403e-05, P55=3.485484926041458e-05, P56=5.1554025828697127e-05, P57=6.172474114873697e-05, P58=7.122593034379812e-05, P59=0.0005968086852905756, P60=0.007392462933806407, P61=0.04252932960045289, P62=0.15623729920089788, P63=0.23656305964129026, P64=0.2794381282258285, P65=0.3865100138372264, P66=0.08581458161295942, P67=0.005908133676349729, P68=0.002179441569170302, P69=0.003533957806470699, P70=0.004306880910218624, P71=0.007735727642826754, P72=0.02773458660826123, P73=0.05054698568221417, P74=0.04950332013688132, P75=0.061225959798481513, P76=0.04853212312705943, P77=0.1724142558964175, P78=0.780849796837456, P79=1.2465264430287328, P80=0.3821187087264328, P81=0.03282944098977891, P82=0.015953428940755784, P83=0.03606891246880494, P84=0.06526517079060412, P85=0.040133957696085604, P86=0.030716923242784126, P87=0.038094761127295554, P88=0.014484133744010043, P89=0.04143518694962787, P90=0.3807229925639036, P91=0.6760066677144143, P92=0.1521206028473871, P93=0.035852299553301076, P94=0.020743904424462892, P95=0.13658055217716764, P96=0.1997379511043628, P97=0.019509068197670634, P98=0.0034351737702674693, P99=0.03612694072151101, P100=0.07489352654620049, P101=0.015731724783826496, P102=0.03406070775935066, P103=0.02514327995256771, P104=0.0010112951482145927, P105=0.008488772823585991, P106=0.07627855627159882, P107=0.08513890880072275, P108=0.22207866668910767, P109=0.08141708018924274, P110=0.0028826733316247473, P111=0.04400752971682342, P112=0.05348280773130146, P113=0.01940524978620859, P114=0.0037310491012014792, P115=0.005399371298454284, P116=0.012077341897042975, P117=0.017384783255413476, P118=0.004930713348521461, P119=0.0026378228262034223, P120=0.010908905878529144, P121=0.014540704326686532, P122=0.0020285534400874272, P123=0.0023399001190423136, P124=0.007878347421127363, P125=0.0035571183381161277, P126=0.0011114903405908517, P127=0.0016928305127380558, P128=0.006818075808268063, P129=0.0030138188155559367, P130=0.0022844980653957815, P131=0.003804660164198973, P132=0.002688984869138859, P133=0.002739053864624259, P134=0.00840301420778795, P135=0.002278638785785555, P136=0.002591426136074806, P137=0.0013831791913432134, P138=0.0011926908259060245, P139=0.0009176491041076217, P140=0.001250681853042879, P141=0.0013954524438161092, P142=0.001058935508786232, P143=0.0011184118001879727, P144=0.0013039423978851356, P145=0.0016730590636608277, P146=0.0007486577692028594, P147=0.00023913510474310982, P148=0.00013776105346410242, P149=0.0001611175461260936, P150=0.0001865228335640021, P151=0.00019792288149274726, P152=0.00019792509260655364, P153=0.0001968650262277586, P154=0.00030497084284302286, P155=0.0001991883661070511, P156=0.00021571316801830394, P157=0.0003255272693584478, P158=0.00031047995606841563, P159=0.0002404847526938194, P160=0.00018939259750834798, P161=0.00019113676737437748, P162=0.00017363081737734883, P163=0.0001322845778688516, P164=0.00010625948925220287, P165=7.535961976715015e-05, P166=6.333154024390952e-05, P167=0.00010114296712308394, P168=0.00021759805764561015, P169=0.0002043592737425718, P170=9.856408079664184e-05, P171=7.425853771794998e-05, P172=0.00011461423733920317, P173=0.00011693463020785044, P174=0.0001328193880684823, P175=0.0002510610348432422, P176=0.00017667108263897017, P177=0.00010493234675590599, P178=4.4189111769911245e-05, P179=4.760701177366493e-06),Row( P0=-618.5108132037668, P1=46.1034924359714, P2=-11.464857153300676, P3=1.4639946565621256, P4=-8.904574706014559, P5=-14.569945986994412, P6=-5.800017782863916, P7=-4.70075513652371, P8=-0.8525502836680933, P9=-3.814509891057298, P10=-8.34717944859234, P11=-3.762195710982513, P12=-3.8160190408290204, P13=-5.521504615742622, P14=-4.429815554857058, P15=-3.3822580131514988, P16=-4.68859068895849, P17=-2.0922275731644278, P18=-5.718865324261359, P19=-0.6907154420676659, P20=-4.309080978435975, P21=0.4213582273246225, P22=0.3666163851231243, P23=1.482163796316342, P24=1.846170173395038, P25=5.671683922961554, P26=5.937735017279932, P27=5.484460831220695, P28=6.426547205197896, P29=4.337629472542995, P30=1.4533539803501523, P31=2.4861826421039983, P32=-1.225422326394047, P33=1.981169621288296, P34=-0.31980346239129903, P35=0.028089277589471152, P36=-1.3812511325242587, P37=-0.9330732073569536, P38=-2.212141621761303, P39=1.4592220431654732, P40=0.6454284732433333, P41=0.6660650737148559, P42=0.702622436807907, P43=0.7450927353737371, P44=0.7549136680249506, P45=0.7019712176361126, P46=0.6289683154050625, P47=0.6158088538794022, P48=0.6203871652354019, P49=0.6051441504789363, P50=0.6192926541521666, P51=0.636576287462408, P52=6.1485429069887205e-06, P53=2.1269807037196808e-05, P54=1.9514441148671636e-05, P55=2.8690202101795726e-05, P56=5.9843765850810574e-05, P57=0.00026638903680365665, P58=0.002074387640195439, P59=0.016180527541514424, P60=0.05406110269272773, P61=0.10907994284661839, P62=0.09675743555129543, P63=0.06795497141536407, P64=0.06671797758170109, P65=0.028833894858193092, P66=0.0049790878165617575, P67=0.004232044254380182, P68=0.01024642888760463, P69=0.023034464541259375, P70=0.031195459989954585, P71=0.044507168985442984, P72=0.03626709605739419, P73=0.03301079268279395, P74=0.024113431414036875, P75=0.030196398959870085, P76=0.0915144658763769, P77=0.15563769291535262, P78=0.08260027210545184, P79=0.041381230795306485, P80=0.020596108891595504, P81=0.024206976275222405, P82=0.02162085928506922, P83=0.02161099136742101, P84=0.03707939684547306, P85=0.016484572458343887, P86=0.006507601450193761, P87=0.018877217344224068, P88=0.03361092854259569, P89=0.02665394746894018, P90=0.05498134121906494, P91=0.0701020222591396, P92=0.0057809460157401986, P93=0.0025247865878469783, P94=0.0051711580850618485, P95=0.007376876287902601, P96=0.011595158241753075, P97=0.08947650339274003, P98=0.0806783102731404, P99=0.003101160207724923, P100=0.002760498635960864, P101=0.0020839714390051118, P102=0.003117308008099912, P103=0.007488590806968066, P104=0.002320100981086836, P105=0.0011361613455460862, P106=0.0017528506947947445, P107=0.006816864280977247, P108=0.014735100814119632, P109=0.0018344846310206237, P110=0.0048935358411951735, P111=0.016226681316158734, P112=0.01664105186163317, P113=0.002249174020655867, P114=0.0034723717660439177, P115=0.002476952996756159, P116=0.0017313721587934627, P117=0.0010138914445798786, P118=0.002017663675733239, P119=0.001179349495362328, P120=0.0018730486946598895, P121=0.005099011107400057, P122=0.0012801409703971378, P123=0.0009666769392208922, P124=0.0013472910840124277, P125=0.002220502439171756, P126=0.0007507358405002411, P127=0.0006349155061302503, P128=0.0025792705337033634, P129=0.0019240467932515849, P130=0.0008465908539475055, P131=0.0015141364275809506, P132=0.0009180083926163769, P133=0.001792870951563174, P134=0.0019465579561640004, P135=0.0031839315564933445, P136=0.002179083589439466, P137=0.0011837667905416792, P138=0.0012334217457436068, P139=0.0005193190388475991, P140=0.0005925036639507075, P141=0.0004948804189809196, P142=0.00047539203178924153, P143=0.0006829792402619013, P144=0.0007255221763028308, P145=0.000820223550405171, P146=0.0002340058662459014, P147=8.380818340612222e-05, P148=0.0001259483369744082, P149=0.0001621603496509906, P150=0.00016862574955264388, P151=0.00020477250839153537, P152=0.0001672053392517518, P153=0.0002418072983741435, P154=0.00028446413878461705, P155=0.00023703493260613764, P156=0.00014807344391427568, P157=0.00018607982851760459, P158=0.00023651340223270324, P159=0.00023187889357291302, P160=0.00020774782295542168, P161=0.00012112708367463422, P162=8.747931253941799e-05, P163=0.00013120958690130018, P164=7.536426720039367e-05, P165=4.3962132576163545e-05, P166=3.9299980829936446e-05, P167=6.0530561110039845e-05, P168=7.864467480769513e-05, P169=0.00018080886000943305, P170=0.00014272504872210107, P171=9.364019482498222e-05, P172=0.00013284165956860432, P173=8.166434589767324e-05, P174=7.130802071279922e-05, P175=7.919000048310644e-05, P176=3.795288501567036e-05, P177=2.9760621317228975e-05, P178=5.045096356100366e-05, P179=6.720473535968505e-06),Row( P0=-659.8718048649067, P1=46.61993681411893, P2=-9.428764393769908, P3=3.122971747978181, P4=-8.256944431060079, P5=-9.179058773344183, P6=-5.562910184533474, P7=-3.3321090478049666, P8=-2.6233763110600563, P9=-3.816081995238152, P10=-6.715558663495434, P11=-5.179871865162767, P12=-3.2096536790080683, P13=-5.7486960187315095, P14=-4.331972365619254, P15=-1.7221421461485715, P16=-5.107370777940109, P17=-2.6167477015464224, P18=-4.14643211873199, P19=-0.47169334209043384, P20=-4.04716251398289, P21=-0.5544164632587567, P22=-1.94778000393415, P23=0.22591411458357546, P24=-1.9569574907471352, P25=0.5404731907281455, P26=0.6886086835651233, P27=1.2156204251645006, P28=3.1920931091653646, P29=5.8277592125319435, P30=3.9899169690370835, P31=4.402418873129725, P32=0.4550838485876178, P33=0.6280573491111896, P34=-0.04231164122691316, P35=1.0838327301991497, P36=-0.44076026857255607, P37=-0.10596702273269709, P38=-1.0767861475441987, P39=0.3851738229521885, P40=0.7482527118714829, P41=0.7676658034351981, P42=0.795952230258509, P43=0.7738538255364592, P44=0.73678515967496, P45=0.7430619051029582, P46=0.7441754173516214, P47=0.7255631628492701, P48=0.739010060461386, P49=0.7426281941842997, P50=0.7370775858479387, P51=0.7282082251955376, P52=3.887934407916366e-06, P53=1.3449847670641963e-05, P54=1.7753035296000217e-05, P55=3.749403310366137e-05, P56=7.62230476369028e-05, P57=0.00044982024837299337, P58=0.0037322441884174887, P59=0.01833929047906919, P60=0.041926933188151876, P61=0.04063879048501898, P62=0.03536964999910835, P63=0.053510157820498484, P64=0.05137246748979939, P65=0.012566979729301712, P66=0.0030288926453913892, P67=0.0031865618870415722, P68=0.009720182083550303, P69=0.015467637733447276, P70=0.013148300859688695, P71=0.00863654212479137, P72=0.006476600160757651, P73=0.005377284211305373, P74=0.007705599712956749, P75=0.0179823271736657, P76=0.05825970155568552, P77=0.07169789818512863, P78=0.019075053887918837, P79=0.00837688417466787, P80=0.003979033554191048, P81=0.002233667386056015, P82=0.0017625279579434432, P83=0.0035020391403917032, P84=0.006903454641506383, P85=0.006234042883057781, P86=0.013702589534056859, P87=0.013737310415991305, P88=0.0156981437672876, P89=0.007576949060309344, P90=0.0013379573058594082, P91=0.0016357763548121877, P92=0.003547618632905193, P93=0.005053943060666005, P94=0.01817746219303104, P95=0.009231814160204287, P96=0.0008957508169326747, P97=0.0012748021459138715, P98=0.001460796324512407, P99=0.0015236197980120212, P100=0.001568789829269739, P101=0.00041425678023169815, P102=0.0010226456932353664, P103=0.002533936992694824, P104=0.002535893906902226, P105=0.0016620658704793675, P106=0.000283856538500863, P107=0.0005952469221337854, P108=0.0031834972997833496, P109=0.004095194652803695, P110=0.0011373629027483913, P111=0.0025142907766256216, P112=0.002837120916139122, P113=0.0007556537075117496, P114=0.0006857154564161582, P115=0.0006006226187454509, P116=0.000634017087148764, P117=0.0008369384567284612, P118=0.0009614967609764213, P119=0.0006084509554550926, P120=0.0004625482831064586, P121=0.0004886286326068198, P122=0.000896424170666048, P123=0.0002890013626029158, P124=0.0003745575390846662, P125=0.0008124769831224319, P126=0.00018429359305780242, P127=0.000305825307890584, P128=0.0032197368545771004, P129=0.0005374444598856632, P130=0.00029871151572289967, P131=0.0006215171569215576, P132=0.0003722264662246248, P133=0.0007916827397859148, P134=0.0006687719620744686, P135=0.0009887653540542134, P136=0.0010888096347747215, P137=0.0007134211611686446, P138=0.000501812399462155, P139=0.00045067472576478955, P140=0.0005479173483632628, P141=0.00026259601081266454, P142=0.00024013083252885035, P143=0.0003156920460761207, P144=0.00033562146404423195, P145=0.00032530079623818324, P146=0.00020777394749713993, P147=0.00011423650465720555, P148=0.00011529200221114901, P149=0.000175188926986609, P150=0.00019799827309456024, P151=0.00018253921873551075, P152=0.0002286805461091588, P153=0.0001577248463231626, P154=0.00015836356999761865, P155=0.00016442596395749946, P156=0.0001407926082090605, P157=0.00014079952813375948, P158=9.455030081126573e-05, P159=9.421855508326242e-05, P160=0.00011004917385630013, P161=9.137759833122905e-05, P162=8.734987616347349e-05, P163=7.788320535424659e-05, P164=6.767287326477278e-05, P165=4.114063241740433e-05, P166=3.726023248966659e-05, P167=3.6306029760312115e-05, P168=4.730469688720204e-05, P169=6.971695093442508e-05, P170=5.9689312362345e-05, P171=2.8494345342637515e-05, P172=3.662784795973312e-05, P173=3.83580473923282e-05, P174=2.7707154151973125e-05, P175=4.5934135511240214e-05, P176=4.0045788232632696e-05, P177=2.748475655584419e-05, P178=1.984876680075966e-05, P179=3.667589009747597e-06),Row( P0=-696.6745628808577, P1=59.24957305737164, P2=-6.99184541930021, P3=4.876066753067451, P4=-5.42505893032565, P5=-14.16480070956072, P6=-4.571675275620829, P7=-1.5210063752034642, P8=-1.0179957907400514, P9=-3.5534946977219533, P10=-7.5281826578036455, P11=-3.6353120549671636, P12=-1.9328013372404311, P13=-7.307837416626988, P14=-4.456040459708374, P15=-1.6635649535591617, P16=-4.73125833816866, P17=-2.821926551115105, P18=-3.8858974235888635, P19=1.0005198247727887, P20=-3.428533849708803, P21=1.3779454171926127, P22=2.033338082063512, P23=2.0248715013293896, P24=0.5040191046007715, P25=0.6094744755556218, P26=-0.5861012467354422, P27=1.3344547927258057, P28=2.5812241076285165, P29=6.460924583660359, P30=3.892820144328063, P31=5.982408067641479, P32=2.094700807021821, P33=3.5597044294170543, P34=1.8612578524208423, P35=0.5240975964027459, P36=-1.0520003956832427, P37=0.40127624955168345, P38=-2.252614023805624, P39=1.552709454309422, P40=0.7490564596147283, P41=0.7306247287311564, P42=0.7119908659460615, P43=0.7025284431145242, P44=0.712423583831408, P45=0.7073657213750802, P46=0.7076585788976675, P47=0.7157951492786757, P48=0.7055049042146253, P49=0.6869271242764666, P50=0.6686284232357645, P51=0.6882958663112725, P52=3.970077447428334e-06, P53=1.5308688643190577e-05, P54=1.407274159431047e-05, P55=1.2594448948553702e-05, P56=1.3623038594099325e-05, P57=0.00022944934992935508, P58=0.006398409122021855, P59=0.027569090969207812, P60=0.035071608524325185, P61=0.023896145021348843, P62=0.023920147930246768, P63=0.03609929709728214, P64=0.020205623732236697, P65=0.003991211642752063, P66=0.0025277934742530058, P67=0.0052088716733660985, P68=0.010688061078380932, P69=0.012676981740941227, P70=0.010016696415433032, P71=0.007374818319342446, P72=0.008150635956163119, P73=0.0055504093366275115, P74=0.012161136396606625, P75=0.02557198688321039, P76=0.033519163024020485, P77=0.031047862386538217, P78=0.018417423037604366, P79=0.01217015307593826, P80=0.009803808704485, P81=0.010194003711720263, P82=0.009416789051137972, P83=0.004933505047463529, P84=0.0075571372382971305, P85=0.009158301372521802, P86=0.011138841184505659, P87=0.012014113290034246, P88=0.008956696563640999, P89=0.00318673991525373, P90=0.0014253731932970077, P91=0.004948883049838726, P92=0.012252355039824853, P93=0.006415509688901048, P94=0.007177404051981115, P95=0.003652736871482674, P96=0.0024931409360546284, P97=0.008543640866075692, P98=0.005747975393935035, P99=0.0012749597908706415, P100=0.000522270677886526, P101=0.00018208525187156016, P102=0.0005474167199768697, P103=0.001015592739398418, P104=0.0006074582731785706, P105=0.0004287226889659041, P106=0.0009222243104727594, P107=0.0006114505775444563, P108=0.0004043287260640001, P109=0.0007300627250741485, P110=0.001736808605979785, P111=0.001793310413033848, P112=0.0014365299498699656, P113=0.001856770921148381, P114=0.0010038300698950087, P115=0.0008554547923563317, P116=0.0012755853193801019, P117=0.0005569087308403689, P118=0.0005234942292469112, P119=0.00041870501789036926, P120=0.00043152474636275233, P121=0.0006843069508764088, P122=0.0019016112047211837, P123=0.0005819070693593811, P124=0.0005113181683641772, P125=0.000677509822079994, P126=0.00038132697576165266, P127=0.00020360527206966873, P128=0.0001562594508817533, P129=0.00016077445649559667, P130=0.00010627387242083982, P131=0.00011961630264009083, P132=0.00018912311791282127, P133=0.0003540785905759418, P134=0.00045029713255499194, P135=0.0006319998140880714, P136=0.0004846226318306769, P137=0.00040188113954397944, P138=0.00033492979580188276, P139=9.16763632219822e-05, P140=9.357285149333455e-05, P141=9.162093691907426e-05, P142=0.00011219488425503094, P143=0.00012553783557224513, P144=6.195605113218733e-05, P145=4.6558915690101744e-05, P146=4.1899064536418784e-05, P147=4.1720998525283495e-05, P148=1.8884280555722254e-05, P149=2.151076779912915e-05, P150=3.3875562062835216e-05, P151=4.894345032112939e-05, P152=4.153028108300725e-05, P153=5.501265174194659e-05, P154=4.946131753532421e-05, P155=7.00268988359176e-05, P156=9.137258507390869e-05, P157=9.777365890956572e-05, P158=9.483257618861695e-05, P159=0.00011514773224795658, P160=8.828147653335634e-05, P161=5.514639358521394e-05, P162=2.7453164081278698e-05, P163=1.6895986593966723e-05, P164=2.8402504368517437e-05, P165=2.181349878798929e-05, P166=4.0456687377295166e-05, P167=4.1642655964343586e-05, P168=3.6321090133975994e-05, P169=3.378057233241529e-05, P170=4.020847716274838e-05, P171=2.3300976334993684e-05, P172=2.4045190649390403e-05, P173=2.9030126473706465e-05, P174=2.0652626631236027e-05, P175=2.536971351111372e-05, P176=3.3272185060858255e-05, P177=2.2737591553541512e-05, P178=8.38343996158903e-06, P179=6.17548474497588e-07),Row( P0=-532.5049644837488, P1=22.10038692424808, P2=-25.173998202289965, P3=-8.262824947080324, P4=-14.456611297858837, P5=-15.317241619477631, P6=-11.099915611790362, P7=-8.813057469083066, P8=-11.068225981466838, P9=-7.511047938073932, P10=-9.780626078733267, P11=-2.946941908852591, P12=3.6113690684564763, P13=5.2643864398742926, P14=4.78363381331121, P15=5.304199716529588, P16=2.615030155912846, P17=-0.17013482166813193, P18=-4.545614272346819, P19=3.8004973045666013, P20=-1.3708202820577178, P21=0.6945126479980619, P22=-1.7607800085922187, P23=-1.4548435929036159, P24=-1.0414286479817134, P25=2.513990179814751, P26=1.457645597448693, P27=-0.026012292043986196, P28=-0.9599699448229266, P29=-0.8717935433749807, P30=-0.26106845635106024, P31=1.7243181693500622, P32=-2.437887732063723, P33=0.2759985344071094, P34=-1.0587672686684622, P35=0.48747749285483516, P36=0.8274167044276267, P37=-1.7803160080239921, P38=-0.41577327023807614, P39=-0.2679794298719468, P40=0.5546417010050725, P41=0.5851745269880755, P42=0.6167085199725831, P43=0.6636487756124833, P44=0.666578912090837, P45=0.6333146401137705, P46=0.5862442014977819, P47=0.5200901854812938, P48=0.4855510802089718, P49=0.49535781075973867, P50=0.5494729345453719, P51=0.5468661853546055, P52=4.0648165417814194e-06, P53=1.120649789544307e-05, P54=1.29976995734293e-05, P55=2.7266241646173123e-05, P56=4.185781105528557e-05, P57=3.0343114948706218e-05, P58=2.5854094991668848e-05, P59=0.00019637549143607292, P60=0.0008579089965487583, P61=0.003855982057736773, P62=0.016151897336527818, P63=0.02395527160518576, P64=0.0207524015575972, P65=0.12746899333750503, P66=0.2824229039419626, P67=0.1810798998154531, P68=0.31927246353073996, P69=0.8380806175199992, P70=1.1631724375708459, P71=1.0421494735611645, P72=0.5729814772822247, P73=0.4074130511298192, P74=0.3478754695791248, P75=0.20326860669606553, P76=0.04412837961894269, P77=0.014172101446389987, P78=0.023263410792127365, P79=0.04580958673691503, P80=0.14222060926315883, P81=0.31374997966067075, P82=0.41943904226736756, P83=0.3532329852102594, P84=0.3159741335399658, P85=0.3102550781861965, P86=0.4640599590704603, P87=0.7100906679662823, P88=0.7928761379912301, P89=0.5300854042433917, P90=0.7340858110684734, P91=0.938256350325288, P92=0.6847556204224388, P93=0.5916676858911224, P94=1.1370119209763743, P95=1.8923208873619022, P96=0.37508935134510135, P97=0.04157735191316195, P98=0.04872867653708208, P99=0.09379056818097543, P100=0.08134578176033017, P101=0.056196048062519204, P102=0.24541729329364148, P103=0.5072619554823032, P104=0.0799957748047311, P105=0.10967987053606643, P106=0.1723691543482786, P107=0.14098721184445637, P108=0.21429573421592576, P109=0.164371409274779, P110=0.12757785342599706, P111=0.1187393214267383, P112=0.12425855455508142, P113=0.05295493456564498, P114=0.046831885920950125, P115=0.0871367249583304, P116=0.11453385029440025, P117=0.05112851942826376, P118=0.21577397836074158, P119=0.10329353242129775, P120=0.09176520702636912, P121=0.06158632018445352, P122=0.013165004993737706, P123=0.006847302478053274, P124=0.009160170660264826, P125=0.01897079106303755, P126=0.010565430651660917, P127=0.008216343363826522, P128=0.014501048102673445, P129=0.017056291891803202, P130=0.011376528167922152, P131=0.014356463519638372, P132=0.023944037772910095, P133=0.02141746012830485, P134=0.013818367474072344, P135=0.013759036240662883, P136=0.015929089312398094, P137=0.00753640294957415, P138=0.004967520875206528, P139=0.0020820576866931013, P140=0.002703349345337679, P141=0.0017811031407437463, P142=0.002920652957122071, P143=0.004124847149977136, P144=0.0033435885625846024, P145=0.0024438337347842316, P146=0.0018949375817409262, P147=0.0016951895090725846, P148=0.0009338571699090478, P149=0.00043317316010065937, P150=0.0004955495286318896, P151=0.000555743082348498, P152=0.0007600349626680603, P153=0.0007034285419154106, P154=0.0006535630404022974, P155=0.00032193352418756726, P156=0.0003488091647473359, P157=0.0005424177495701893, P158=0.0011427961251106928, P159=0.0010477090869011431, P160=0.0005957281938331398, P161=0.000411674592664956, P162=0.00024513374474979994, P163=0.00022361033494874561, P164=0.00030011764605346916, P165=0.0003237690191258433, P166=0.00019137153747967944, P167=0.00026561656985038486, P168=0.0004700474668258037, P169=0.0004658963849247285, P170=0.0003439200696310717, P171=0.00031387049167950224, P172=0.00038791649537484464, P173=0.00044068633564749805, P174=0.0004051375235944376, P175=0.00024067038668696173, P176=0.00019952178068833953, P177=0.00018099069148524722, P178=7.121312248246904e-05, P179=8.910960126413244e-06),Row( P0=-680.0775911270282, P1=43.822555638959365, P2=-4.07573859140982, P3=-0.6472292277394391, P4=-7.579348924994414, P5=-9.54946512495454, P6=-8.326171357826707, P7=-5.0613525257676, P8=-1.304306005106075, P9=-5.14644512763773, P10=-6.328962017963497, P11=-4.188866213212642, P12=-4.221978891471002, P13=-6.953211156297047, P14=-3.9871480563026527, P15=-3.187684532938536, P16=-5.105275540779472, P17=-4.0036104989008505, P18=-5.592358882543049, P19=-2.045679964980044, P20=-5.096045553679236, P21=-1.256671596128995, P22=-1.8013691095834632, P23=1.2478621010440776, P24=-1.0803374347505637, P25=2.150849694110318, P26=0.0039530583731173, P27=-0.3524453168169023, P28=2.2822244839551282, P29=3.8958286436319614, P30=1.9598322473198178, P31=4.997037993246888, P32=0.29206833328794857, P33=-0.008945658546858878, P34=-1.4718732071032405, P35=-1.6297324320623854, P36=-0.8323558797428832, P37=0.4257355228588009, P38=-1.1066844853860303, P39=-1.06778509439129, P40=0.7556643197643991, P41=0.7639299272086925, P42=0.7734048691290416, P43=0.7305691043775788, P44=0.7594173051410873, P45=0.7841365504754529, P46=0.7287273498192177, P47=0.6826928540463703, P48=0.6966628169326586, P49=0.6992460962392867, P50=0.6660363202006132, P51=0.684948842242728, P52=2.6116705487054732e-06, P53=6.1167976668507805e-06, P54=1.2767895133452672e-05, P55=2.3414552143016133e-05, P56=5.475650094127902e-05, P57=0.0004934083351158411, P58=0.0028984321369430075, P59=0.01745059046220049, P60=0.03668578771443533, P61=0.05199108146733967, P62=0.0956611903040961, P63=0.05121352099196619, P64=0.007828743903392887, P65=0.0016819884522863192, P66=0.0012580887295587384, P67=0.0038703642399930977, P68=0.017591688036032598, P69=0.019091906421527433, P70=0.016738301515189395, P71=0.011405373205267168, P72=0.021043760898284112, P73=0.027430846666210613, P74=0.024994708405396058, P75=0.016685743717944464, P76=0.011950585415742605, P77=0.013648238093092423, P78=0.009935885138581381, P79=0.010867171461881242, P80=0.005672993972752939, P81=0.0033900917738410707, P82=0.004544879953489586, P83=0.017354094983722812, P84=0.03687372594381617, P85=0.03783460808770267, P86=0.023890056753627282, P87=0.024509593666784218, P88=0.014503520062703838, P89=0.0017530232994319513, P90=0.0006226383552627169, P91=0.002531910536813301, P92=0.009181826090542593, P93=0.009145233370406996, P94=0.02034082233132714, P95=0.005886842837476708, P96=0.0003002576455745739, P97=0.0012731901555011061, P98=0.0023495219511298813, P99=0.0036359028687394834, P100=0.003722369507872046, P101=0.0015673064826139537, P102=0.0010422171722563398, P103=0.0010115694769589159, P104=0.0004751335956893895, P105=0.0011505894459385342, P106=0.002292695588020873, P107=0.0017754686905369066, P108=0.0012935347987246995, P109=0.0024132779296599668, P110=0.0018615990695922202, P111=0.0032453259606705282, P112=0.0016332857425305448, P113=0.0015447029656650841, P114=0.0011856097952509664, P115=0.0034054348610751597, P116=0.0022657458578526856, P117=0.0021461550787538163, P118=0.0014692893902562376, P119=0.001557645327223399, P120=0.0012824869901731134, P121=0.0009079918511292633, P122=0.0007703357179631728, P123=0.00035822433256181856, P124=0.000247419201088355, P125=0.00014934610736895733, P126=0.00014615748789897342, P127=0.00025268646817149117, P128=0.00015640762849815942, P129=0.0001713279161162693, P130=0.00016097428620104968, P131=0.00025517906495330995, P132=0.0002973824240492829, P133=0.0004161979842932519, P134=0.000407396236479124, P135=0.00034846791266193884, P136=0.00016988648033043917, P137=9.128559622076313e-05, P138=6.958777738935269e-05, P139=8.282774519270783e-05, P140=9.8004165825343e-05, P141=0.00018694451198548038, P142=0.0002538324803023579, P143=0.0005402989809591513, P144=0.0002625636660357512, P145=0.00013190694951418556, P146=0.00021856759419144214, P147=0.0001547270892643923, P148=0.00016902415134191287, P149=0.000255206531907819, P150=0.00036406354822616713, P151=0.00033058021289353507, P152=0.0004538403259884861, P153=0.0004459757109463452, P154=0.00047697388227270043, P155=0.0004638873930298681, P156=0.0003538806750173396, P157=0.0003744253694123449, P158=0.0005543045932641826, P159=0.0005390124600445454, P160=0.0005247397363927108, P161=0.00047729864073748555, P162=0.00048072923420502446, P163=0.00033828972203668985, P164=0.0003381663561826658, P165=0.0002142086722752522, P166=0.00018788288845376135, P167=0.0001823206334439658, P168=0.00018555952628262383, P169=0.00024193146339175636, P170=0.00016967268018224633, P171=0.00010135946772198562, P172=0.0001950526755199845, P173=0.00021593571625219538, P174=0.00021743256220363598, P175=0.0001594302120450878, P176=0.00010958129709499484, P177=7.072511466996059e-05, P178=4.186026585756089e-05, P179=6.098952782644633e-06),Row( P0=-720.1307807041994, P1=59.21715570652329, P2=-4.384289979212273, P3=4.069465281313992, P4=-5.520772062321763, P5=-11.865005801230888, P6=-4.545684750451506, P7=-2.7125703024387167, P8=-1.7176136744282438, P9=-2.543152624021582, P10=-4.936021098633554, P11=-5.08019150821957, P12=-1.5506845364520212, P13=-7.490611713445826, P14=-4.4992872719426344, P15=-3.257439035421936, P16=-4.012174036866451, P17=-4.096428981599913, P18=-5.311332404750195, P19=-0.36398765583979426, P20=-3.4444973140809294, P21=0.26770121943991204, P22=0.025808256274889887, P23=1.6376503581687358, P24=-0.45021694174845, P25=-0.9103088471590969, P26=-1.8070434990259925, P27=0.32134083754600695, P28=0.6929754721676527, P29=3.6836757526902706, P30=1.5699652327034317, P31=3.7730939677689026, P32=-0.6834241017928219, P33=2.915013542358368, P34=2.9680780524944494, P35=2.731645342271151, P36=1.432311951427062, P37=1.4340864729077192, P38=-1.7481327709761076, P39=0.4974586856199237, P40=0.7531347792054596, P41=0.7289227315797082, P42=0.7240562439009688, P43=0.6793772281991745, P44=0.6948352497426608, P45=0.6972782167024048, P46=0.720241361901372, P47=0.7192304603898658, P48=0.6870767734165077, P49=0.6946622706592037, P50=0.7145610002332998, P51=0.7347896234471527, P52=2.8953953045202647e-06, P53=1.1126836806735106e-05, P54=9.802100388683715e-06, P55=1.1123700706905785e-05, P56=2.3750043062538362e-05, P57=0.0010690035591828862, P58=0.009920304731263653, P59=0.02111326494507643, P60=0.019704449384906567, P61=0.021827715187441924, P62=0.01706799728737291, P63=0.03295013774635777, P64=0.017564618485356644, P65=0.002231730200794331, P66=0.00294142698739976, P67=0.003922132981936435, P68=0.009397671635363394, P69=0.005432257038721595, P70=0.0029451389344023267, P71=0.010877703100368124, P72=0.005791683087039273, P73=0.0027105230866205574, P74=0.0042531456899218695, P75=0.012881291652082577, P76=0.017619576828544512, P77=0.015832950413976993, P78=0.006838332958319898, P79=0.00339040032654634, P80=0.005948701732966499, P81=0.015521835223526237, P82=0.009066456413704446, P83=0.0023765187878267676, P84=0.003691755801346091, P85=0.006298416495855764, P86=0.006253296192308731, P87=0.018441525260774725, P88=0.018155786690146936, P89=0.003922490809228092, P90=0.0014190949821397127, P91=0.002008251046182797, P92=0.0023067446252770323, P93=0.0008938664206994468, P94=0.00213817118741992, P95=0.00361965620599007, P96=0.007523479389294965, P97=0.0031092504919274485, P98=0.0010346637864809113, P99=0.0007338170041509562, P100=0.00042396094680086186, P101=0.00023742120488241683, P102=0.0012504820917072024, P103=0.003081386492965355, P104=0.0010871378791687946, P105=0.000565395722933948, P106=0.0008410699467768099, P107=0.0003624964328215496, P108=0.001440690108023205, P109=0.0012636541852654492, P110=0.0003804970519017943, P111=0.0003829319732397923, P112=0.0010542792798465565, P113=0.004293566656653889, P114=0.0024742395231478072, P115=0.00044022543050812414, P116=0.0005210758178770786, P117=0.0010414030241216931, P118=0.0009458835769421116, P119=0.0002713120558414313, P120=0.00030344999530895793, P121=0.0014505969416705082, P122=0.0003126578936351367, P123=3.05754910477769e-05, P124=5.964671881263186e-05, P125=5.63288870141524e-05, P126=0.00012865115246220567, P127=0.00017133085332106984, P128=0.00025150332326787046, P129=0.00011137555888607033, P130=0.00010716495069584666, P131=7.562092535496622e-05, P132=0.00017207244524447126, P133=0.0001136561332123758, P134=0.00021345077074758748, P135=0.0001888056326282081, P136=0.0001540707318722666, P137=0.00011373716442963626, P138=5.862364676741799e-05, P139=3.4978650071255204e-05, P140=3.415344716921915e-05, P141=2.50979987915538e-05, P142=3.0060383020604444e-05, P143=2.8736882287257777e-05, P144=2.665258000640737e-05, P145=2.1042979074420417e-05, P146=1.0854115745789245e-05, P147=1.7857754178801687e-05, P148=4.027826090980347e-05, P149=1.2788712217658112e-05, P150=2.2741890725575347e-05, P151=5.4657220049369014e-05, P152=3.865720919740666e-05, P153=4.288301118834585e-05, P154=1.968431669442418e-05, P155=5.408860601516731e-05, P156=3.881324048756832e-05, P157=4.5058925735037185e-05, P158=4.357212052417203e-05, P159=4.637288672382819e-05, P160=3.4708385227066264e-05, P161=2.837092283909351e-05, P162=1.7611492654813725e-05, P163=1.613183602510045e-05, P164=1.814862361370827e-05, P165=2.362978905985424e-05, P166=1.4520925589032573e-05, P167=1.4780746882793402e-05, P168=1.1577698008646162e-05, P169=7.632915018607386e-06, P170=1.017698539457544e-05, P171=1.0054976079198079e-05, P172=9.872125246353387e-06, P173=1.0110877511319208e-05, P174=1.0318848185013429e-05, P175=9.997048236241937e-06, P176=1.5116076075806071e-05, P177=1.1769601326040362e-05, P178=5.129130862011147e-06, P179=5.459235265734483e-07),Row( P0=-631.1901333347056, P1=40.66749305330895, P2=-12.639355293075623, P3=-2.2208156473224108, P4=-11.801425672977853, P5=-16.842762230880545, P6=-7.774621093169067, P7=-6.047442641767436, P8=-5.013625669133465, P9=-3.9974799515879993, P10=-12.384019080700169, P11=-5.055437954459592, P12=-6.312936229077727, P13=-7.593575420190433, P14=-6.144926253633435, P15=-5.880151315781557, P16=-3.3984885135385285, P17=-2.9627035135262862, P18=-4.012046165098636, P19=6.328539281263475, P20=2.462439684157909, P21=7.115151465350854, P22=6.32061326675435, P23=5.30446947028894, P24=0.1928484862013042, P25=3.3550701571623884, P26=-0.7604797751456811, P27=0.7643542785168582, P28=1.6295368172742457, P29=2.060916607911716, P30=-1.5329405613190479, P31=0.45058062744110766, P32=-3.9733196933499872, P33=-0.7421044052898031, P34=-1.221751168484429, P35=0.09722493744948703, P36=-0.29959680226487945, P37=-0.2657979856369262, P38=-1.6264771881031856, P39=1.5339596547734646, P40=0.5480143599096814, P41=0.5555561246958536, P42=0.5899718380833697, P43=0.5882546048780497, P44=0.6132911970821713, P45=0.6348616233973219, P46=0.6361652342373625, P47=0.6180527222119679, P48=0.613743125296412, P49=0.5972548628803449, P50=0.5472093887607076, P51=0.5290651779379254, P52=2.9183831504316848e-06, P53=1.065380537211227e-05, P54=9.143575318603427e-06, P55=9.656238068986573e-06, P56=1.5798996246312226e-05, P57=7.648077916210875e-05, P58=0.0006248117844397494, P59=0.004455275232214824, P60=0.022775900235326683, P61=0.07568161042161348, P62=0.14925200071698935, P63=0.22221356584303423, P64=0.17342131872802866, P65=0.08322728303111794, P66=0.024955022307712026, P67=0.005347569879043787, P68=0.003303978313880039, P69=0.00818285970031594, P70=0.015029376888819453, P71=0.015825795973825135, P72=0.020026561833064775, P73=0.022393480684111716, P74=0.04172114583659265, P75=0.06642675669072058, P76=0.07331427626528673, P77=0.1013625149470073, P78=0.12526188339529667, P79=0.1437954665369422, P80=0.08334734137983733, P81=0.0365781734318317, P82=0.028700366498913433, P83=0.04012255288590574, P84=0.06315624314985055, P85=0.0617465478414949, P86=0.07816573303494972, P87=0.106194457820913, P88=0.1293051812901157, P89=0.05498762796017355, P90=0.029731061294642813, P91=0.029720585121102518, P92=0.03287851436433637, P93=0.012950075806498105, P94=0.013991708181408054, P95=0.02805945454428845, P96=0.037442804300918724, P97=0.03287878502035868, P98=0.01418487781649258, P99=0.0057527457716425295, P100=0.006918259107382055, P101=0.004463695450635809, P102=0.006374144749034983, P103=0.007395940160950805, P104=0.00517576491357357, P105=0.0039458911864064805, P106=0.002157709797471816, P107=0.0019731040793749726, P108=0.006764135651622475, P109=0.010060153429769193, P110=0.006572305235558724, P111=0.0058794278611957705, P112=0.00484365678484454, P113=0.011616138023480082, P114=0.011449022002710623, P115=0.008279503973030311, P116=0.0071501614272071485, P117=0.005423452045992262, P118=0.004158031603459646, P119=0.004503891175553136, P120=0.006396757958069838, P121=0.0027261535675052423, P122=0.0013740180114380617, P123=0.0014744044419305155, P124=0.000801091655997877, P125=0.0008679978665578031, P126=0.0004622288862044823, P127=0.00021472999809397884, P128=0.00045830191248549595, P129=0.000804027919521737, P130=0.0006426756487510143, P131=0.0012926963325292974, P132=0.002020232322287424, P133=0.0019030237445932868, P134=0.0019984524264530447, P135=0.002155445855889233, P136=0.0010532723705291142, P137=0.0004563823753331408, P138=0.0004222266403961538, P139=0.00024236267057692505, P140=0.00034605319924354286, P141=0.00035917732066490497, P142=0.0005605675033047734, P143=0.0007233764787966482, P144=0.0006784802571863566, P145=0.0004973558989999438, P146=0.0001363104770083942, P147=7.21632661293711e-05, P148=9.185032576598812e-05, P149=0.00017999360716291745, P150=0.00018426218829918923, P151=0.00027818252663548584, P152=0.0003136203806061387, P153=0.00040986045246956947, P154=0.00036897671633139164, P155=0.0005145898973382437, P156=0.0005257243221731228, P157=0.0003538376890052661, P158=0.0003684255944313789, P159=0.00028334471345001493, P160=0.00023851415187526784, P161=0.0002436364512881987, P162=0.0002706140073586011, P163=0.0002147470017138368, P164=0.00020846998374953008, P165=0.00022110574362589234, P166=0.00020717884607821793, P167=0.00023439601940342719, P168=0.0003918031343646575, P169=0.00025253597076758786, P170=0.00012921853664964959, P171=7.086090427499021e-05, P172=0.00010673554791254198, P173=0.000175717576557258, P174=0.0001791013648113385, P175=0.00015552433260989087, P176=0.00012600895790188898, P177=0.00010897359625246384, P178=5.2815237335224764e-05, P179=6.889421917051928e-06),Row( P0=-542.2701709062004, P1=23.83226788349451, P2=-23.856534640735088, P3=-11.241928904227915, P4=-15.106482635170856, P5=-15.603720390729235, P6=-10.091332759855, P7=-8.439737449465685, P8=-10.935336814364447, P9=-5.22264198845777, P10=-13.761726292668175, P11=-3.8582144926125523, P12=-0.8022770393678996, P13=-0.7956160145465008, P14=2.8374243107267296, P15=3.1476720870090205, P16=4.7883763370709875, P17=3.3735508866312047, P18=-2.5141637674495194, P19=3.742469174490665, P20=-2.2037171492685834, P21=-0.17578899775284396, P22=-1.4319958907213872, P23=-2.073411638493934, P24=-2.384695583483651, P25=1.3206767403700381, P26=-1.2166220713224716, P27=-0.433198247995046, P28=-0.3253479178532679, P29=-0.9055311435317484, P30=-2.2409533090461045, P31=1.1449636832787795, P32=-2.667297276317663, P33=0.5284869942223457, P34=-0.2301375688130952, P35=-1.4491223625365104, P36=-0.605505191526619, P37=-0.19253055354684054, P38=1.2195427691789726, P39=2.5491631044234113, P40=0.6398002813170698, P41=0.6707409875396598, P42=0.6679895373108161, P43=0.6745716708539309, P44=0.7527194545252798, P45=0.7398192571407218, P46=0.6427947088938056, P47=0.6099359522666193, P48=0.6096782157114999, P49=0.6269284392454726, P50=0.6294984856897419, P51=0.6150620485213351, P52=6.474250672584572e-07, P53=2.4163077263896046e-06, P54=6.155348511788631e-06, P55=1.6594645511511025e-05, P56=3.887751365899268e-05, P57=0.0002468835879148176, P58=0.0008462256186636882, P59=0.0016594339110500467, P60=0.0047077963772376, P61=0.012169600527342308, P62=0.022624365472121537, P63=0.03423763286418878, P64=0.08556611862854326, P65=0.24549227339275506, P66=0.2576075671204885, P67=0.26546427002282336, P68=0.46085953794614076, P69=0.6434177033021304, P70=0.5422561902976006, P71=0.5300507137119059, P72=0.3888388369446259, P73=0.146427702605461, P74=0.028084468912777587, P75=0.016957852987796056, P76=0.02740882503595012, P77=0.11636855765178092, P78=0.45461006627293943, P79=0.8102757975561131, P80=0.45628769270768743, P81=0.228130892795302, P82=0.19191991658101967, P83=0.353697146617447, P84=0.7718822320296745, P85=0.5596150508014291, P86=0.3902528753624344, P87=0.4087644588917179, P88=0.4315788942543994, P89=0.3932925353832875, P90=0.5483858798479285, P91=0.8822799386383302, P92=1.5180724778745334, P93=0.39274918982813894, P94=0.16633915874203434, P95=0.21194113819356536, P96=0.5700848482523275, P97=0.689323540393329, P98=0.22775023661482713, P99=0.04976094143870073, P100=0.05034511418611336, P101=0.045336695939780056, P102=0.07276976594534006, P103=0.17421841800484314, P104=0.26424224185465106, P105=0.16688233129943453, P106=0.07919343835963566, P107=0.03205644242636981, P108=0.022060541066946222, P109=0.02681673212589226, P110=0.05122053448001788, P111=0.08029872239710364, P112=0.11626818188371639, P113=0.13515134423792916, P114=0.08720493405711946, P115=0.0729582781283269, P116=0.05391396535385515, P117=0.062465936140206096, P118=0.15624927098096714, P119=0.020297872234982833, P120=0.00816568058728755, P121=0.0074316407189835895, P122=0.017400483571841513, P123=0.014143133383123513, P124=0.018845483956760705, P125=0.009293172247796251, P126=0.0025367725630034083, P127=0.0023099765149067, P128=0.007851812030008773, P129=0.01061311399560612, P130=0.0036344249682092006, P131=0.006997006256070244, P132=0.013977947682135675, P133=0.012460982593717964, P134=0.016956492973622086, P135=0.01015805704830665, P136=0.022853230650495306, P137=0.009420051266906886, P138=0.0017145215455324498, P139=0.0020194883159390825, P140=0.002638802977044027, P141=0.003641522076277702, P142=0.006006455036858813, P143=0.005824343188743402, P144=0.0036897241921505213, P145=0.001792664536428467, P146=0.0013492742519989772, P147=0.0012049668613906216, P148=0.0007700198049823071, P149=0.0009538983377634335, P150=0.000800252577050622, P151=0.0013982304728880589, P152=0.0013666740428283216, P153=0.0009447428956882849, P154=0.0005889217646892267, P155=0.0008632702948996963, P156=0.0008997929150014709, P157=0.001026624291828233, P158=0.0012004394872693772, P159=0.0015961649023133937, P160=0.001480521632928753, P161=0.0014987149186050294, P162=0.0011451687794051494, P163=0.0011780134103781077, P164=0.0012936390094722056, P165=0.002343555789192001, P166=0.001217487614556645, P167=0.0010714722475065246, P168=0.0010105451908775868, P169=0.0008147471359445389, P170=0.0007422628461865342, P171=0.0006728157003169924, P172=0.0005564112804373378, P173=0.0006539146342539425, P174=0.0006374829255640067, P175=0.00048046547978436617, P176=0.0003586523944365871, P177=0.00030951849825468187, P178=0.00015595069090038432, P179=2.4512440092446252e-05),Row( P0=-699.8061463733992, P1=62.21724798146973, P2=-3.2125193396199094, P3=4.9177905513352, P4=-3.1823138395795434, P5=-10.901398341667162, P6=-2.162838191631995, P7=-4.729219734611292, P8=-2.545858445229803, P9=-1.8459188597748935, P10=-5.412372888753984, P11=-3.534937738002117, P12=-1.915216335648882, P13=-6.230915299811857, P14=-4.777795310688093, P15=-2.2265559757063125, P16=-4.173991732518197, P17=-1.5389149142825072, P18=-2.705580357139231, P19=1.2765245861767054, P20=-3.6958527854695182, P21=-0.6932736607186777, P22=-0.36083561469262865, P23=0.5154088439220137, P24=-0.9767211229712933, P25=0.9543814779517646, P26=-0.3506074338106117, P27=2.205395024987322, P28=2.885625442196501, P29=4.827559766067414, P30=2.09032801129302, P31=5.361170429745062, P32=1.5950709469755382, P33=5.4782030404268545, P34=4.122964240147487, P35=3.07658592101336, P36=1.0566542089997768, P37=2.1222276925555237, P38=-0.05749928371726265, P39=2.0884561898179115, P40=0.7530645902314032, P41=0.7168684237837621, P42=0.7171019737341864, P43=0.7186681648993887, P44=0.7317525358642615, P45=0.7011959839536783, P46=0.6752849183924362, P47=0.6830977384815956, P48=0.6972445928737016, P49=0.7124811796329487, P50=0.7269294911034835, P51=0.7836167877224115, P52=5.900515309857354e-06, P53=2.136631651053703e-05, P54=1.0862388879621583e-05, P55=6.828950119275877e-06, P56=2.5600140780951918e-05, P57=0.0009169393820161402, P58=0.00906515469652567, P59=0.022965320115839474, P60=0.03174519314258753, P61=0.04253309455367667, P62=0.02923919322900159, P63=0.010175819920512719, P64=0.009076411323957109, P65=0.0028691956889003085, P66=0.0022633632478074213, P67=0.0036370305925075845, P68=0.005484195073830633, P69=0.010773857348963817, P70=0.011226528300857226, P71=0.017257022038742083, P72=0.014810883332410112, P73=0.004665947742197941, P74=0.0026783311999806535, P75=0.0033135014832417113, P76=0.012985542969830254, P77=0.030198837225617725, P78=0.019611371117812665, P79=0.016082433842131847, P80=0.01385688431553344, P81=0.0226961038498807, P82=0.030602276453905182, P83=0.007747562518504339, P84=0.0025823234900463642, P85=0.0021101453955284202, P86=0.002426958939731564, P87=0.0033969851574538273, P88=0.004566571128780901, P89=0.003817574163211654, P90=0.0022709668562942626, P91=0.004097874080281021, P92=0.006256128946810311, P93=0.00358898083885927, P94=0.0037084919502913603, P95=0.003150202915838847, P96=0.0025648753618507814, P97=0.006138346097542078, P98=0.0057041957730528375, P99=0.001324800786376274, P100=0.0007390551460041715, P101=0.0002560628203999214, P102=0.00021027686477588801, P103=0.000705194479344031, P104=0.001373443080589075, P105=0.0010281664251865822, P106=0.0016344274434414705, P107=0.0006035105838104761, P108=0.0005975232649787962, P109=0.0008995439051180057, P110=0.0017177449794705296, P111=0.0011604402123611553, P112=0.0008233257473108572, P113=0.0011270735447955, P114=0.0008270255860632039, P115=0.001026157736460347, P116=0.0008695710120311461, P117=0.00032850931232819087, P118=0.00034764063729487944, P119=0.0005655685723082593, P120=0.0001490197399457036, P121=0.00025353473310829583, P122=0.0010897937595764355, P123=0.000333704443440363, P124=0.0002798747063982241, P125=0.00028108746506453925, P126=0.0004247727059129387, P127=0.0005463021901686652, P128=0.00021116300087231592, P129=0.00038519776548429926, P130=8.977982790116877e-05, P131=0.0001714382428549592, P132=0.00016934801853284626, P133=0.0005035254039874325, P134=0.00038522458929536114, P135=0.0004582579844302487, P136=0.0002140079671226226, P137=0.0001290300005995576, P138=0.0001181784189260864, P139=6.143691941208821e-05, P140=8.565641573755413e-05, P141=4.9297112683688355e-05, P142=7.102609608749966e-05, P143=8.854930232507694e-05, P144=9.361646845399491e-05, P145=5.45700773836728e-05, P146=2.523805432674051e-05, P147=1.6538292949165946e-05, P148=1.4878653248828808e-05, P149=8.316443718160697e-06, P150=1.4331277544907626e-05, P151=2.2827447588905894e-05, P152=3.265515647591994e-05, P153=2.725909600974757e-05, P154=4.3702739873369896e-05, P155=3.758331804060342e-05, P156=2.581220115271489e-05, P157=2.6724186918092383e-05, P158=2.165034978065819e-05, P159=2.724687904080294e-05, P160=2.2822908930501007e-05, P161=1.89233816984652e-05, P162=1.58836559080676e-05, P163=7.17005885390786e-06, P164=1.0206923306570128e-05, P165=1.4468861811714953e-05, P166=1.3684632436437128e-05, P167=2.338195402372215e-05, P168=2.921772783211382e-05, P169=3.453688785622785e-05, P170=1.614553119832361e-05, P171=1.3110701730350162e-05, P172=1.5492958285548942e-05, P173=1.3138620332990608e-05, P174=8.611556069915812e-06, P175=7.596923626495003e-06, P176=9.656167928277394e-06, P177=1.1829534572785184e-05, P178=4.451162833489632e-06, P179=3.7158911868702227e-07),Row( P0=-671.400324225733, P1=58.836365646444584, P2=-9.171185430942227, P3=3.695838446220106, P4=-6.246328671644828, P5=-13.28510867695713, P6=-6.486863792953819, P7=-4.678247974963709, P8=-4.250988271791904, P9=-5.316263525687702, P10=-9.98896999862395, P11=-5.571737963686528, P12=-2.085055119030566, P13=-9.9898610220063, P14=-4.8816893387614115, P15=-3.375435836905843, P16=-5.375440444354364, P17=-3.406757310195873, P18=-4.530165994898796, P19=0.6550392098211586, P20=-5.819029667917054, P21=0.5870218353317671, P22=-1.0891415017511035, P23=0.5396952705450416, P24=-0.6751552332471609, P25=0.11103153990898876, P26=-0.06872821608758034, P27=1.4493036136635737, P28=2.7048435198757845, P29=5.489256643151698, P30=2.3665420121999436, P31=4.706024737201505, P32=0.1368073233495161, P33=2.7048306455254343, P34=1.7746447074904577, P35=2.106379396719434, P36=-0.4668635860085483, P37=0.29222102581269715, P38=-1.8996785790575088, P39=2.094779947640871, P40=0.7348830183253908, P41=0.697629580511626, P42=0.6942094150738187, P43=0.6882883918466038, P44=0.7092047443163569, P45=0.711505302311604, P46=0.7028085048387445, P47=0.6977287421016942, P48=0.6893916492142652, P49=0.6962563701061634, P50=0.7027442214748928, P51=0.7261315940112285, P52=1.5312470727079814e-06, P53=7.149896436598987e-06, P54=1.2131905186480246e-05, P55=1.1450432468257856e-05, P56=1.782191673448318e-05, P57=0.000615876345401397, P58=0.011168406163145196, P59=0.03714296230108697, P60=0.05557101993849817, P61=0.041148032207873685, P62=0.024352344150310064, P63=0.018754139177609732, P64=0.02093581089901189, P65=0.007990011893051065, P66=0.00448350796365399, P67=0.0070720260557125166, P68=0.015341513204775371, P69=0.02579851908824066, P70=0.015450982240046877, P71=0.01156178086244675, P72=0.00857153576413365, P73=0.0045933607646098835, P74=0.005713140313521856, P75=0.009550844600200198, P76=0.022847457706547492, P77=0.049703262540680304, P78=0.03810535374490857, P79=0.019262805128132804, P80=0.010844950337960204, P81=0.01407266659752887, P82=0.012400231316493367, P83=0.007369541014689901, P84=0.006956651212519261, P85=0.009350627533664118, P86=0.018441771991339933, P87=0.015058058176042376, P88=0.015491087155503543, P89=0.01374354090045315, P90=0.010574944345475967, P91=0.015102785891996111, P92=0.023833730043118447, P93=0.04133332948704597, P94=0.03591039829007017, P95=0.035851256476560914, P96=0.013791631457472368, P97=0.006527653169137127, P98=0.0028885346850953944, P99=0.004111153043499283, P100=0.0036686150936306797, P101=0.0017998119078790895, P102=0.001459522468691246, P103=0.006083566576190551, P104=0.005808919603171911, P105=0.003310979183960842, P106=0.001724787296751778, P107=0.001162022042679201, P108=0.0020576352543548184, P109=0.00316977277356219, P110=0.005300670883981882, P111=0.014295457475176186, P112=0.004901175772555583, P113=0.004138086553773775, P114=0.004451213922354798, P115=0.0022946436897726174, P116=0.0012334851769060576, P117=0.0022695628307057045, P118=0.003950608305312398, P119=0.0024196243857017587, P120=0.001692070618158565, P121=0.002089787715458787, P122=0.003934000774237991, P123=0.0009413695681139534, P124=0.000746050928151261, P125=0.0006296932073795786, P126=0.0003900141253138763, P127=0.0001759349096564239, P128=0.0002239889672097814, P129=0.00017436547757783497, P130=0.0001349290818878055, P131=0.00020658986260823714, P132=0.000503912249701475, P133=0.0005666622839538969, P134=0.0010181213690146325, P135=0.0011497483267541172, P136=0.0009552357445285486, P137=0.00047708906782572435, P138=0.0002855989510132499, P139=0.00016191196894990703, P140=0.00018177470475505826, P141=0.00017925998102261397, P142=0.00033123315039453526, P143=0.0004082595539946697, P144=0.0001497025782136791, P145=5.752956544713225e-05, P146=2.7439470917096817e-05, P147=2.827415960304468e-05, P148=3.464184522706294e-05, P149=2.6047090557406107e-05, P150=2.988011209360585e-05, P151=4.74204034767625e-05, P152=5.163862508010369e-05, P153=8.28001206760698e-05, P154=6.251439674583632e-05, P155=6.824524136892964e-05, P156=0.000108832703117529, P157=0.00014688902346168855, P158=0.00012400932320418353, P159=0.00010254595679404016, P160=7.751632842872517e-05, P161=5.827781299042078e-05, P162=3.589443037058574e-05, P163=2.9753125333053026e-05, P164=2.3516138073263174e-05, P165=2.5192000799880648e-05, P166=2.6910293438698848e-05, P167=3.7299230474695816e-05, P168=3.287989658178026e-05, P169=2.953335398690363e-05, P170=2.956462508416064e-05, P171=2.9864528920864515e-05, P172=2.534726022877639e-05, P173=2.696450679111962e-05, P174=1.9749263888038618e-05, P175=2.0276408812208e-05, P176=2.5514424185953015e-05, P177=2.1383619509936215e-05, P178=6.93706054551771e-06, P179=5.079684313093048e-07),Row( P0=-533.128743949018, P1=22.226401272534225, P2=-23.041194541045567, P3=-7.4396803350467735, P4=-13.344134799811313, P5=-14.898650722589128, P6=-10.374585869142873, P7=-8.34719299076003, P8=-10.12056346347049, P9=-6.7648048016642415, P10=-10.56818947529631, P11=-2.523030974603536, P12=3.2965302884937424, P13=5.038426941249066, P14=7.512130004178214, P15=6.607533976242421, P16=2.3898547008595754, P17=-0.08487702843866164, P18=-4.933936163574031, P19=2.799237500933567, P20=-1.655294857529726, P21=0.4976888437687394, P22=-1.0865057700397587, P23=-0.8770477245120444, P24=-1.5126669474841268, P25=2.4431290575107676, P26=0.4888408814339508, P27=-0.05115295502857886, P28=-0.4299018458380299, P29=-0.21580579882993176, P30=-0.045772309088965867, P31=1.7921521992020562, P32=-2.4485589600975115, P33=1.0023837843941372, P34=0.31432797575317145, P35=0.4260492956992015, P36=0.13418578905638673, P37=-1.7279968284653566, P38=0.05764318239322986, P39=1.3872345599710922, P40=0.6822119949010595, P41=0.7068456204283304, P42=0.7337488791365984, P43=0.778948732770383, P44=0.7713149543265203, P45=0.7249022435329635, P46=0.6703930771353782, P47=0.6381972671751713, P48=0.6100521012164655, P49=0.5923980439007376, P50=0.6206722856707808, P51=0.6676480053623266, P52=4.447033425052898e-06, P53=1.4230691559277201e-05, P54=1.091033204788943e-05, P55=1.3772418847140142e-05, P56=2.6321852338760844e-05, P57=5.338897714867901e-05, P58=0.00010065009381092858, P59=0.0005263560077454627, P60=0.001774928763109515, P61=0.0037371647215252513, P62=0.014508824130853047, P63=0.023373211373568067, P64=0.017014079924734186, P65=0.03253312212532522, P66=0.14194962394315963, P67=0.3192428695476383, P68=0.48454833270909115, P69=0.7017914091315546, P70=0.8592863353943778, P71=0.9040764777348439, P72=0.4977937023382822, P73=0.4002353689880475, P74=0.3362552788985411, P75=0.17396323266431846, P76=0.029770416019326878, P77=0.005147667488958233, P78=0.006991513151288621, P79=0.009083458099632812, P80=0.015293846133874989, P81=0.029799019776086524, P82=0.085123795553926, P83=0.22456076739480804, P84=0.5290770144292577, P85=0.6592272207421027, P86=0.6706613122844546, P87=0.7483842296293379, P88=0.839208686559307, P89=0.5763305585170331, P90=0.7365353562773085, P91=0.8319786188915602, P92=0.6494353340394361, P93=0.5074295031094689, P94=1.0866520803772781, P95=1.758740129428504, P96=0.2391048862233422, P97=0.08244834119536096, P98=0.08515997064428338, P99=0.055536399171537305, P100=0.05370554266611585, P101=0.028366658061180314, P102=0.1313315564022597, P103=0.6331743256897039, P104=0.15655001359149498, P105=0.09808119380014067, P106=0.11978158059844736, P107=0.17713312751871113, P108=0.19049431788900903, P109=0.08697533411791152, P110=0.07600543732633416, P111=0.08350139475873679, P112=0.13350019247181136, P113=0.07236914198311098, P114=0.036463012596790176, P115=0.05572192990716801, P116=0.08620039770294308, P117=0.0578344477674473, P118=0.11581634313158758, P119=0.05293828803458591, P120=0.040284378810170086, P121=0.03752645102018714, P122=0.014742616397288608, P123=0.009446001260514276, P124=0.0107841678346227, P125=0.011267109845358015, P126=0.005509679110101758, P127=0.007103846440933864, P128=0.0074399232032811334, P129=0.011701275538590614, P130=0.00874841793788905, P131=0.009777481418701274, P132=0.015354576989496237, P133=0.01400007321846476, P134=0.007917011768935195, P135=0.009973796042260903, P136=0.011253273121519589, P137=0.008002111564117944, P138=0.0027562999431164246, P139=0.0019343207781033973, P140=0.0025584336334520076, P141=0.002109789521605637, P142=0.004290127496712713, P143=0.0053146273019065585, P144=0.003719504002604602, P145=0.0029123743675264656, P146=0.0008330349180707197, P147=0.00045475332523946927, P148=0.0004413136382558122, P149=0.0003647390971547894, P150=0.00048601282855703247, P151=0.00038507094472216806, P152=0.0005120277141502792, P153=0.0005310643252698748, P154=0.0005870550567686726, P155=0.00033683603797846393, P156=0.00038518304514381706, P157=0.0006121934613161564, P158=0.0007787977847574319, P159=0.0006874881127643589, P160=0.000601669219357624, P161=0.00042994684624690876, P162=0.0003137390368915083, P163=0.0002670465193864989, P164=0.00026716774279029586, P165=0.00023741817942598048, P166=0.00021458490059057553, P167=0.0002524847496776809, P168=0.00031478675879662883, P169=0.00026107593674166947, P170=0.00020706531020646282, P171=0.00034655769699405833, P172=0.000416073779188167, P173=0.0004711280892859592, P174=0.0004937727411604392, P175=0.0004057434997567853, P176=0.00023069817011619654, P177=0.00023553453354447897, P178=0.00022870485492361558, P179=3.137462046800249e-05),Row( P0=-611.1260589112251, P1=42.739025565883175, P2=-8.761312999029796, P3=-2.7287914652256573, P4=-11.566261576570005, P5=-14.19975864287302, P6=-12.158220463030576, P7=-5.734630941973902, P8=-5.921408733706919, P9=-3.0555531048351976, P10=-12.055457903106447, P11=-6.369938484815523, P12=-4.407056596589866, P13=-9.603772957807037, P14=-5.454828633084768, P15=-4.202387632669225, P16=-3.6123019201526114, P17=-0.6144134531030315, P18=-2.55517826625719, P19=2.7032277483573735, P20=-0.2849505327156645, P21=2.7152712549724853, P22=3.007021835778934, P23=4.023352899841109, P24=1.6950589794287017, P25=2.8793567996047438, P26=-0.9611679808028549, P27=-1.0352131496933277, P28=1.783582963332609, P29=2.1842481598108674, P30=1.1752168613854197, P31=3.1570033367067145, P32=-0.5004543126122152, P33=0.19963121552880964, P34=-2.3160475791039086, P35=-1.3446301485966194, P36=-1.0749277071401449, P37=0.27381186776547806, P38=-0.8662749341174082, P39=-0.11572007803414587, P40=0.572319771742382, P41=0.5702375653431523, P42=0.5524996277347213, P43=0.5658306418340129, P44=0.6281457123452772, P45=0.6659817923880621, P46=0.6331387467171474, P47=0.6140851529590245, P48=0.5964005969765344, P49=0.5843725457127481, P50=0.5865128540770786, P51=0.5840610591708946, P52=3.874974417695776e-06, P53=1.0942838201469103e-05, P54=2.469292057781383e-05, P55=3.8181864021393494e-05, P56=6.539993750845047e-05, P57=0.00023855256010607014, P58=0.0019706510322628507, P59=0.016995301149669323, P60=0.04281669963849345, P61=0.056343127042913325, P62=0.16643052316111953, P63=0.18420671744055175, P64=0.0974902286306795, P65=0.09172163092466307, P66=0.05386996004947261, P67=0.027746960335214562, P68=0.020299571277133437, P69=0.01719519697663397, P70=0.010360476007994671, P71=0.00990696966423399, P72=0.016338162312592948, P73=0.03918627450858818, P74=0.09871897252710078, P75=0.07978507497440067, P76=0.05070543500549347, P77=0.08389784127118363, P78=0.09753051140772456, P79=0.06391212660257455, P80=0.03811346129055677, P81=0.03389044433677118, P82=0.027776174792552184, P83=0.027195574040283537, P84=0.039779240370625434, P85=0.05248993864211602, P86=0.042465321963840945, P87=0.038586601569789214, P88=0.03361633788036412, P89=0.014848067133918908, P90=0.010009165023862463, P91=0.014330588011241626, P92=0.015752102613163274, P93=0.11359327270391757, P94=0.14888595929609472, P95=0.09162457213073275, P96=0.029254310976663558, P97=0.014417661486002141, P98=0.012040291774973639, P99=0.006779863262247834, P100=0.01629752359121397, P101=0.004512324719850677, P102=0.0018030146588283935, P103=0.0029216156771044726, P104=0.0018256085322191386, P105=0.008964227527366257, P106=0.018149438623404363, P107=0.009675842528724074, P108=0.012731162558297562, P109=0.00898928754630716, P110=0.03418343521223244, P111=0.030332024815472966, P112=0.0067349208476732755, P113=0.005460629362126909, P114=0.02200978979826685, P115=0.02500997004248427, P116=0.013324664395701187, P117=0.0032328220175231615, P118=0.0027955671312013142, P119=0.006071697761829594, P120=0.004589052615402084, P121=0.004924161220754543, P122=0.005177802590668471, P123=0.001414914974614619, P124=0.002037051813058992, P125=0.006313035850764773, P126=0.002810191294725739, P127=0.0008235148215696827, P128=0.0008916651466230333, P129=0.0003921384846671694, P130=0.001370043980659415, P131=0.0012886344184028836, P132=0.002276882840337187, P133=0.0020437848965133556, P134=0.003439770051552208, P135=0.0035466671051956155, P136=0.0020400488956415063, P137=0.0013916493283395478, P138=0.001198464957466253, P139=0.0012439122530171415, P140=0.0012084110482716648, P141=0.001837793975735865, P142=0.00157014344693676, P143=0.0014582428036415268, P144=0.001472964900803187, P145=0.0011190740494410462, P146=0.0005171258657795943, P147=0.0002930279702602566, P148=0.0003033440194954558, P149=0.00048675114539888656, P150=0.0009020616993438585, P151=0.0011478540968045733, P152=0.0011732521285520027, P153=0.001558306714236633, P154=0.0011457994409453825, P155=0.0012461344940008603, P156=0.0011977057415548764, P157=0.0015596411465201023, P158=0.0022637730973507244, P159=0.0027509578701052973, P160=0.0027702695270605815, P161=0.0035156342843526913, P162=0.0034934282430885023, P163=0.0030351630720613114, P164=0.0026978256312488397, P165=0.0031116336166060843, P166=0.002851720979876412, P167=0.0026409101511233033, P168=0.003026398246463349, P169=0.002168805713582984, P170=0.0016312494894175438, P171=0.0010793048269959339, P172=0.0010036760326245417, P173=0.0007989181292874253, P174=0.0005518727973775006, P175=0.0006167915511008303, P176=0.0007688383509166277, P177=0.0007175753031549314, P178=0.0003203681661825227, P179=4.2946381142485417e-05),Row( P0=-565.5990237623855, P1=29.849509079836974, P2=-22.036528932870755, P3=-8.124536763588386, P4=-15.914164459534037, P5=-16.05895990321613, P6=-9.45909052567967, P7=-9.330766386967381, P8=-9.097425728383008, P9=-4.76861424544699, P10=-13.41198062187946, P11=-3.9427568991729043, P12=-0.6728020604148138, P13=-1.3126078823237564, P14=3.7892452603696785, P15=4.256301697125665, P16=5.273172462676046, P17=3.931496860931853, P18=-1.6572541745994072, P19=4.245179902697193, P20=-1.563513524617416, P21=2.160129626649535, P22=0.9440058628934362, P23=-0.8765438482886122, P24=-3.9675240947102175, P25=0.37198660357390845, P26=0.2269362001757599, P27=1.493128995348709, P28=0.7144869741889194, P29=-0.538795490356274, P30=-1.5810653011513232, P31=2.1346041349836957, P32=-1.8825814908089582, P33=0.09929299756506609, P34=-1.3281401356775338, P35=-0.1927929629806469, P36=-0.43810689214282156, P37=-0.4510343084016758, P38=0.16027683056975428, P39=2.0086459392978795, P40=0.6909460071698837, P41=0.7020457828392596, P42=0.7333606079418848, P43=0.7614413368675352, P44=0.7247492997864992, P45=0.6587918784274174, P46=0.628138993634219, P47=0.6428459683542389, P48=0.6767401221133621, P49=0.6655381197809309, P50=0.6703120849983828, P51=0.6636368064952571, P52=5.460569472747123e-06, P53=1.5083730086965287e-05, P54=1.1661242361136235e-05, P55=2.1032093950637768e-05, P56=3.25128448634261e-05, P57=4.865788467089204e-05, P58=0.00012267827392111158, P59=0.0008879914109539059, P60=0.005527188759023194, P61=0.013798269493891315, P62=0.0318471872454561, P63=0.07967727986829755, P64=0.09135979639138106, P65=0.13579554212319908, P66=0.2027365510969121, P67=0.190566403286067, P68=0.39376599034803994, P69=0.6667203135795009, P70=0.6860693467588983, P71=0.4774879096740638, P72=0.11928857664964418, P73=0.032441673209185, P74=0.015214068521140666, P75=0.03954389319279257, P76=0.1677768499613698, P77=0.31501374552458816, P78=0.27012687638486477, P79=0.2815545619456155, P80=0.22334800224815748, P81=0.18472430596131717, P82=0.17602509377312703, P83=0.20190456204713078, P84=0.3992903990721282, P85=0.41231618716577245, P86=0.4301393406227257, P87=0.4126023707061026, P88=0.48814784214365453, P89=0.49066385392547074, P90=0.5440056477106897, P91=0.6384429134982685, P92=0.305834690151163, P93=0.19786352748098546, P94=0.2260635601804879, P95=0.22963146522348754, P96=0.16405688081289307, P97=0.19480309904886334, P98=0.2531616874888568, P99=0.15010278950993924, P100=0.05282463629599495, P101=0.051396670146861986, P102=0.09110015075747531, P103=0.1400609937711332, P104=0.08507000928760929, P105=0.02852856423198215, P106=0.03403425187883396, P107=0.022760484841672347, P108=0.044610701112084074, P109=0.06616977715549761, P110=0.10148539526036936, P111=0.09719765523709581, P112=0.05791322883834263, P113=0.07042641727011738, P114=0.08731945434077105, P115=0.05231275259088426, P116=0.021605282444511622, P117=0.019456904565853417, P118=0.013722469559325703, P119=0.02721380507154249, P120=0.04502581721185277, P121=0.012045927256740245, P122=0.010673192320276427, P123=0.01778791263195791, P124=0.008083630037612491, P125=0.00256077889675866, P126=0.0010984723562348264, P127=0.0022749461064430253, P128=0.006798276403555547, P129=0.0074468836188797734, P130=0.0018852361461287623, P131=0.005086139783420559, P132=0.01846375970703806, P133=0.006948088875936952, P134=0.010479961470013526, P135=0.02124142725576152, P136=0.01928749086654062, P137=0.0021345692328545085, P138=0.0060886450200917765, P139=0.003043186754644224, P140=0.003487286122882446, P141=0.005702627661648478, P142=0.012038404560026071, P143=0.0077977179896904095, P144=0.0025150348120782516, P145=0.001970191047179363, P146=0.0010801611870872201, P147=0.0013416866628920485, P148=0.0008696205050959743, P149=0.00037946349975173394, P150=0.000752411589192854, P151=0.0007156871398028346, P152=0.0009647173674477978, P153=0.0008271708583983946, P154=0.0009591521447770862, P155=0.0006376899691057503, P156=0.0005651521266624046, P157=0.0008894311491831883, P158=0.0010090056803654895, P159=0.001291053506345068, P160=0.0011525418391325991, P161=0.0009191575745005893, P162=0.0008143240448673458, P163=0.0007762741853508962, P164=0.0006454322573680487, P165=0.0005779559091395598, P166=0.0005774345341751846, P167=0.0008739151163130712, P168=0.001097107333380628, P169=0.0006303564536143699, P170=0.00037708388461971375, P171=0.0004475983412589173, P172=0.0004685046331819692, P173=0.0005194095614548983, P174=0.00036640835634179297, P175=0.0002573335418008792, P176=0.00022936405196594134, P177=0.00022202868202687542, P178=0.0001221328281068218, P179=1.2535258993106364e-05),Row( P0=-542.5483352779979, P1=29.442927223996456, P2=-24.515543818540376, P3=-5.986834303181563, P4=-15.45001038902041, P5=-19.692684867660997, P6=-8.581093145237766, P7=-6.09840399752548, P8=-6.702250896995496, P9=-6.94419948743513, P10=-11.66385005654123, P11=-7.554089535751553, P12=-6.068359992695711, P13=-11.306761103233372, P14=-4.857900511763557, P15=-2.79504203717299, P16=-3.238441120960377, P17=5.804101929726821, P18=2.651988059677449, P19=11.484766489751213, P20=5.333462039609167, P21=6.624290255049193, P22=5.234051814283265, P23=3.344939486222092, P24=-0.4602709613330965, P25=3.8105070901221136, P26=-1.3735797292016518, P27=-1.3172233696581304, P28=0.07972127083255084, P29=-1.7234648174085336, P30=-2.6041845501255527, P31=-0.07375705838141164, P32=-2.7793309119448275, P33=0.04678399002138915, P34=-2.773416851190057, P35=0.20524206992618302, P36=-1.5700598957599188, P37=-0.3768398067653612, P38=-1.325580264117062, P39=2.265479490425108, P40=0.6573755667414708, P41=0.602398426369396, P42=0.5913414726646418, P43=0.6220705380726917, P44=0.648725506192136, P45=0.6521349221595719, P46=0.6822489828997201, P47=0.73738562837179, P48=0.7213893002636429, P49=0.7372594931881391, P50=0.7502097177365651, P51=0.6951563483655464, P52=5.6456342975889505e-06, P53=1.0483183612287507e-05, P54=1.687631080426718e-05, P55=2.992925145288003e-05, P56=3.090490358363267e-05, P57=2.451729184086613e-05, P58=2.9434449110609504e-05, P59=0.0009372346731702374, P60=0.01600031638143656, P61=0.0644392640814704, P62=0.271383226263762, P63=0.6551346744805394, P64=0.6949664282361351, P65=0.7049606463564565, P66=0.4941774756685376, P67=0.23990836283383216, P68=0.04812992677778609, P69=0.010162727277234794, P70=0.015096477685987748, P71=0.03799755540826072, P72=0.07363736654122206, P73=0.07361192092354948, P74=0.14312269266399053, P75=0.2326585636458267, P76=0.362982323474748, P77=0.5779289140390218, P78=0.7366970233195667, P79=0.807545211161342, P80=0.6519193280915481, P81=0.45182536056015515, P82=0.6381244867127256, P83=0.7159725626047224, P84=0.43286153303958835, P85=0.1876115558354181, P86=0.16023768825967136, P87=0.2506457700235547, P88=0.2785552846299159, P89=0.2836690776492513, P90=0.42499658110020555, P91=0.39102499607513774, P92=0.30505572140003445, P93=0.2169731387323933, P94=0.24654070111347068, P95=0.2916563706455006, P96=0.12114427883643976, P97=0.11566736312053562, P98=0.3642246671708254, P99=0.2019041230125365, P100=0.09881835246354884, P101=0.03951398979229114, P102=0.032338468523487575, P103=0.06133133763989113, P104=0.13008648909210496, P105=0.1086839331501768, P106=0.07063297448247148, P107=0.043779002399123475, P108=0.056191924579214236, P109=0.0436903862590472, P110=0.10540086309589453, P111=0.10280701294551432, P112=0.041483855879553194, P113=0.023384077509684634, P114=0.02515965689935163, P115=0.041482835732256265, P116=0.03532391446306747, P117=0.024248692084137505, P118=0.016013847125846248, P119=0.013880351007751729, P120=0.017128359191662362, P121=0.03131958011281007, P122=0.026954195159626465, P123=0.02401403790847725, P124=0.012801734107723088, P125=0.010631969265102877, P126=0.0073145111015731215, P127=0.0057041338801331635, P128=0.0061297360789074095, P129=0.0031593620024840956, P130=0.004961965280952507, P131=0.007206057166627745, P132=0.010798820048294553, P133=0.0073665315254529615, P134=0.013209245208315165, P135=0.01371595553064966, P136=0.010427533290277896, P137=0.009428774865875867, P138=0.0036818556320522677, P139=0.004006664541975738, P140=0.0034187544754369133, P141=0.005114657606682518, P142=0.004628527747872532, P143=0.007303322935832594, P144=0.006150255671996866, P145=0.004104923556011884, P146=0.0014425313535434176, P147=0.000882164559820438, P148=0.0005771661681948487, P149=0.0008003993012075429, P150=0.0010159198810236617, P151=0.0012962286257279223, P152=0.001797325975443933, P153=0.00152386096612431, P154=0.0018161099989794966, P155=0.0022868960400964852, P156=0.002065002469619894, P157=0.0021143369197931783, P158=0.002609531064803688, P159=0.0024177910448385668, P160=0.0020824611054488583, P161=0.0019802653431733675, P162=0.0017534373081927734, P163=0.001535680297187712, P164=0.001152495297713911, P165=0.0009668478391927888, P166=0.000972645128312458, P167=0.00095456975500974, P168=0.001087706687262269, P169=0.001099839648405016, P170=0.0012339066402124162, P171=0.0016084623596397181, P172=0.0012770617962091207, P173=0.0010219357230173535, P174=0.0011783979164195386, P175=0.0017100910213398582, P176=0.0013380451935129758, P177=0.0010415890571951777, P178=0.0005182825758563277, P179=5.3043391753014164e-05),Row( P0=-657.3553547199865, P1=50.27929708206315, P2=-8.150325633258065, P3=4.092997305765012, P4=-5.859736109026719, P5=-7.0505757557371735, P6=-7.7647662160807664, P7=-2.261410804752288, P8=-0.0985265063463608, P9=-1.3776327609225925, P10=-5.537567452937251, P11=-4.353127491683842, P12=0.5212066315390339, P13=-4.8448558983613, P14=-3.9115940578664676, P15=-0.1748762734979516, P16=-4.4098410304342, P17=-1.8421236293327696, P18=-5.090766842758442, P19=-1.489736792012963, P20=-3.2705570719962633, P21=-0.7895014185466592, P22=-2.707541047320819, P23=0.5003996945947796, P24=-1.1341674037146094, P25=-0.19387170338509385, P26=-1.0422750269393435, P27=-1.2455892831611026, P28=3.018028182967578, P29=5.431178043128671, P30=4.711326623674029, P31=6.909694761054989, P32=0.20868738182545646, P33=0.9151481453159729, P34=-0.26518816416232216, P35=0.04353464882053733, P36=0.7893160054390324, P37=1.3160446748556505, P38=-0.5025860257582881, P39=0.3333765406881524, P40=0.6903191087929083, P41=0.7466956096645945, P42=0.7866731564133371, P43=0.7146764893967603, P44=0.6977367567503079, P45=0.7678966466256482, P46=0.7772004181991362, P47=0.7327304692021257, P48=0.725438984192241, P49=0.7079689302212758, P50=0.6926599538244594, P51=0.68709954513994, P52=6.62382367476454e-06, P53=2.340191722271516e-05, P54=3.85487097425964e-05, P55=9.02206552725924e-05, P56=0.00016826178312539763, P57=0.0008083143468582536, P58=0.006353457204753966, P59=0.029547674201494848, P60=0.058658628816805225, P61=0.03290653382722596, P62=0.03651459545747253, P63=0.05717393276675114, P64=0.022349875548797363, P65=0.004237797920194372, P66=0.002463968336957313, P67=0.0033376861380639816, P68=0.02540234884760391, P69=0.03233919204402203, P70=0.013130343664488, P71=0.00555150767662053, P72=0.004184022773253392, P73=0.005377111406898444, P74=0.0194600425601397, P75=0.038532031661093175, P76=0.032348644317141864, P77=0.034112803529695986, P78=0.021718589155249553, P79=0.013280492613419919, P80=0.006591377343710557, P81=0.0034520782476404814, P82=0.0026849974989398193, P83=0.002686334892583751, P84=0.006654001857109171, P85=0.014845715532719995, P86=0.030665511977584905, P87=0.028116505984064788, P88=0.006669796154119819, P89=0.0009455451790833986, P90=0.0003914511764880563, P91=0.001327328651922985, P92=0.009123418963087632, P93=0.027083816168642205, P94=0.01258489330932395, P95=0.0020864991597917154, P96=0.0006652471162896688, P97=0.002323287703869053, P98=0.004790392494154313, P99=0.005673784486144462, P100=0.0012486467300606282, P101=0.0005450505564034084, P102=0.001780512106904656, P103=0.007130050634652399, P104=0.005768477621141732, P105=0.0011112422541139852, P106=0.0008632733493544813, P107=0.003914620394650565, P108=0.006662128016447869, P109=0.0017792978935865951, P110=0.004068555004124847, P111=0.017035216351813702, P112=0.0025793167324731953, P113=0.000619351580395815, P114=0.0012996330354431616, P115=0.0007567239475695767, P116=0.0005790781351167524, P117=0.0006937757231026373, P118=0.0007494945884440759, P119=0.000903841848724304, P120=0.0006767641383272978, P121=0.0006489741013532378, P122=0.00039076395431943, P123=0.00041617199245750486, P124=0.00044121815529740634, P125=0.0005017363827808602, P126=0.001331217712867734, P127=0.0017695767674724432, P128=0.0003898853825617963, P129=0.0003552701604987004, P130=0.00035046361156051617, P131=0.00039875205331509474, P132=0.0007769471087629545, P133=0.0004182665879050692, P134=0.0005949624691716835, P135=0.0007623951308994452, P136=0.0010283511107892042, P137=0.0007309398714704093, P138=0.0005573862909812628, P139=0.0013827746773651226, P140=0.0013190968561833834, P141=0.0012644610980129141, P142=0.000789554475195014, P143=0.0004235480169587969, P144=0.0006997574328106578, P145=0.0003776049069621796, P146=0.00013146570804321677, P147=0.00010963820754836867, P148=0.0001632091112237476, P149=0.00021875694277683676, P150=0.0004128303414728688, P151=0.0004142419083190237, P152=0.00040195834200020333, P153=0.00046387032949287495, P154=0.00036284289621938434, P155=0.0002728483086371154, P156=0.00024685928728617785, P157=0.00027106201496554803, P158=0.00020758378570037012, P159=0.00022023190443046597, P160=0.0002131372084747511, P161=0.00016553017008093205, P162=0.0001465930637084539, P163=0.00014479956312030004, P164=8.918453623156786e-05, P165=6.685327035939609e-05, P166=5.287513330605052e-05, P167=5.285192799857161e-05, P168=0.00012062817784535334, P169=9.763086483509652e-05, P170=6.776649288341517e-05, P171=7.852140221178958e-05, P172=0.00010175887874524693, P173=8.307459049980949e-05, P174=9.58309457782005e-05, P175=9.324350855390263e-05, P176=5.65194819669781e-05, P177=4.216047565869289e-05, P178=3.2628204190793986e-05, P179=3.509008486455844e-06),Row( P0=-680.904160835539, P1=39.34252821278815, P2=-6.526820571573362, P3=0.6793985368360896, P4=-7.135840578453551, P5=-8.886115475798718, P6=-8.25020344120115, P7=-2.746390666567845, P8=-0.9722774987829701, P9=-4.692236304758108, P10=-5.665224585718153, P11=-3.8845089888846998, P12=-3.336931352666699, P13=-7.023743809523818, P14=-4.953138771477417, P15=-2.6590070861898574, P16=-5.309115378417508, P17=-3.9934592753361327, P18=-5.006932579264041, P19=-3.344870125325249, P20=-4.050382445154649, P21=-1.0633183788199447, P22=-2.740378368125351, P23=-0.7845021579830167, P24=-2.6222570612268066, P25=-0.3972565258866837, P26=-1.1828242535126874, P27=-0.6811579995836581, P28=1.1335340946745305, P29=2.759302380797274, P30=0.3456774580251582, P31=1.3163243776683093, P32=-1.2727991520468211, P33=0.08152511933918372, P34=-0.04437894251246594, P35=0.7483361704379775, P36=-0.7680944751648319, P37=-0.23734145427226963, P38=-0.7386166627286023, P39=0.7779430857950782, P40=0.7331376983180226, P41=0.7170878458454727, P42=0.7477574632488635, P43=0.7536809246802513, P44=0.7266631018893228, P45=0.7527423387322708, P46=0.7249843499385427, P47=0.7254191821026831, P48=0.763942471888477, P49=0.7383565992371361, P50=0.7146850279275261, P51=0.7156072680416707, P52=1.5189147465875985e-06, P53=5.4432856841513785e-06, P54=2.247050529799393e-05, P55=5.476055390608613e-05, P56=0.0001407247686710779, P57=0.0008754861259197681, P58=0.004666886729416718, P59=0.01681672268332659, P60=0.02427361089073952, P61=0.017521025411472147, P62=0.05301097855588612, P63=0.07861671283474571, P64=0.03572261256930773, P65=0.0093251089832817, P66=0.002501290925478746, P67=0.0038561499109861248, P68=0.006871524009834668, P69=0.00766220984780257, P70=0.005668907116590392, P71=0.002865070931615576, P72=0.0019815213990115707, P73=0.0026242755329698667, P74=0.004171202995874118, P75=0.0061890972081527496, P76=0.01287394356358171, P77=0.02821300721114414, P78=0.025943116873162187, P79=0.022929973969785702, P80=0.010058386065193707, P81=0.003505689635937721, P82=0.001908794481694854, P83=0.0023874374232877747, P84=0.004288304001563434, P85=0.004479723449039897, P86=0.006395957393644632, P87=0.010487995320050519, P88=0.014784131194634535, P89=0.015895383484499554, P90=0.005082717101642879, P91=0.0005983143461605087, P92=0.0006595209911819226, P93=0.0006516329181876939, P94=0.001083599999067826, P95=0.0009744852003086258, P96=0.0006785769044369012, P97=0.006959811021079737, P98=0.011527526753211576, P99=0.0011652358834148978, P100=0.00028378825549711244, P101=0.00034323161426833443, P102=0.0007972811554256821, P103=0.000658495828041029, P104=0.0015111514336230462, P105=0.0023303487293736044, P106=0.00039847005693765424, P107=0.00021072272914121254, P108=0.0007060741592070602, P109=0.0015229769783147394, P110=0.011618126332460833, P111=0.012795030489441552, P112=0.0005669539133342016, P113=0.0006672331595368138, P114=0.0016535963247911562, P115=0.002996615571045364, P116=0.0010137079375120836, P117=0.0009886271401803204, P118=0.001505009256413686, P119=0.0013983375466380145, P120=0.0005776948863047436, P121=0.0008019537565937518, P122=0.0006562031426708611, P123=0.0005842405298692058, P124=0.0004970382394172804, P125=0.00039062304050092845, P126=0.00027024316245258365, P127=0.00015851965107932417, P128=0.0002360237316102065, P129=0.00020807550249551682, P130=0.0001980150259586488, P131=0.00047993174038930424, P132=0.0003494615120412752, P133=0.00018243561104108505, P134=0.0002828695039068424, P135=0.0003153067207382941, P136=0.0003523124374027093, P137=0.00024243937710477614, P138=0.00015985672815676912, P139=0.00012149829292476957, P140=0.00010102315108850318, P141=0.00011067006296526597, P142=0.00011224952737338533, P143=0.00015115304517169395, P144=0.000175627041152193, P145=0.00012308199392506, P146=8.555584894619159e-05, P147=0.0001253472853600869, P148=0.00010137339577303481, P149=0.00014610499710200424, P150=0.00025218480898589186, P151=0.0003326735734274868, P152=0.00033274369463549, P153=0.000367154026629278, P154=0.00037820850507237747, P155=0.00024022789090189094, P156=0.0002938117442691429, P157=0.000305873618920471, P158=0.0004563582869133384, P159=0.0006508832034075036, P160=0.0005135768281515549, P161=0.0008208314374565755, P162=0.0008512031534818119, P163=0.00043946048120259223, P164=0.0003777870488160103, P165=0.00031276143844373916, P166=0.0001772719061785542, P167=0.00025208564491589395, P168=0.00019879242920293482, P169=0.00015936399545777715, P170=0.00011411608397493012, P171=0.00010124756660510866, P172=0.00015612367308289897, P173=0.00013453566183555988, P174=0.00010107647919451943, P175=9.259780428726418e-05, P176=4.2576385211366006e-05, P177=6.506976959584522e-05, P178=7.351267338932714e-05, P179=1.2771506963622409e-05),Row( P0=-678.0613657244604, P1=54.42279715447633, P2=-8.891001374047484, P3=2.182408407429629, P4=-7.604303556309393, P5=-13.11760721476577, P6=-4.288224409728924, P7=-4.993701790032788, P8=-4.424630429774446, P9=-3.506489582075767, P10=-8.721929210881353, P11=-5.16755533542067, P12=-3.468250500989251, P13=-7.236450789088786, P14=-5.282349230647493, P15=-3.435250523449795, P16=-5.52087486751333, P17=-2.6874256193669264, P18=-4.32946042188344, P19=1.3956852957736894, P20=-3.821932700315496, P21=0.045902954492722484, P22=-1.6757434432081018, P23=-2.175856283983268, P24=-2.2747263781107683, P25=-1.277273358842111, P26=-1.6777523996607917, P27=-0.11202115880298981, P28=0.706326557192937, P29=4.100330131924202, P30=1.7937073219642403, P31=5.448095105974842, P32=2.283017814140357, P33=4.823280854052171, P34=4.114478180296223, P35=1.5704236553315085, P36=0.6405920985145531, P37=0.43607593507913983, P38=-0.5022705455635406, P39=0.27665352164213397, P40=0.7617939015272085, P41=0.7177953341911081, P42=0.6873206355576872, P43=0.683448463811908, P44=0.6641620892430605, P45=0.6600550419881568, P46=0.6644218541532265, P47=0.6323424319740365, P48=0.6534773564381452, P49=0.6678586853955707, P50=0.6703845866534509, P51=0.7040601225614639, P52=1.704072117015206e-06, P53=7.3557034586917136e-06, P54=1.2006791966529816e-05, P55=1.7377363848534424e-05, P56=2.7035469257722118e-05, P57=0.0006627562463895616, P58=0.015590356326547372, P59=0.04854977597872909, P60=0.03660804373819473, P61=0.020452477477765192, P62=0.013125068540341883, P63=0.012240061149085655, P64=0.02120868340982451, P65=0.0173878847649904, P66=0.010584730127073618, P67=0.01688902598983153, P68=0.017265299108069142, P69=0.012850994527245188, P70=0.009101457622138244, P71=0.005254054346906876, P72=0.0022969633352564168, P73=0.0023070614766578044, P74=0.006180188228192069, P75=0.00952278614390949, P76=0.017774422220062456, P77=0.05312814554874259, P78=0.06188942283235561, P79=0.0408829922284029, P80=0.016210515551547285, P81=0.007242558615556925, P82=0.005491072248792061, P83=0.011066267409562654, P84=0.018061679486863536, P85=0.012163031700025844, P86=0.0043546990880149175, P87=0.006022846467644173, P88=0.013142505615667878, P89=0.0087899024835452, P90=0.0073737983749406185, P91=0.004280920797029618, P92=0.010739058562828488, P93=0.017355636017938006, P94=0.020636296898816648, P95=0.010991940652057733, P96=0.003982731440640922, P97=0.0022115938675710835, P98=0.007052596846846361, P99=0.0031379116407903615, P100=0.0009148804076867114, P101=0.0005139155165157328, P102=0.0010093881542145094, P103=0.0025824919641770964, P104=0.0023235049413514466, P105=0.0018586032655976009, P106=0.0011023848866590527, P107=0.0010000895864028607, P108=0.0008735620649242094, P109=0.0010410162016313749, P110=0.0036656001785450444, P111=0.004828712056341536, P112=0.003043164981045574, P113=0.0034168172933956836, P114=0.004985700335947374, P115=0.0031339662843415527, P116=0.0009781550005620021, P117=0.0008869449138712772, P118=0.0010503787004754394, P119=0.0007275111661019378, P120=0.0009152136551512071, P121=0.0005538560839517269, P122=0.000484363787004475, P123=0.00032602861940659293, P124=0.0002623749470907359, P125=0.0004365556563097849, P126=0.0002951154397102687, P127=0.00021565839450820003, P128=0.00017802364393698427, P129=0.00015817138435464121, P130=0.0001090181231330127, P131=0.00013901577859264336, P132=0.00023800375398863594, P133=0.0003704959219041897, P134=0.0005140382738232246, P135=0.00044178484626945493, P136=0.0004214061873465906, P137=0.0002670675687695835, P138=0.00017564580350364135, P139=8.315032225037271e-05, P140=9.717676738546486e-05, P141=0.00011315182977446786, P142=0.00013273138151740784, P143=0.00013963528435217065, P144=6.26216250539192e-05, P145=3.848293773039994e-05, P146=6.258269847827852e-05, P147=4.657395770216492e-05, P148=2.8305340925753764e-05, P149=2.7652978659278026e-05, P150=5.003983804283032e-05, P151=5.537891610687765e-05, P152=5.238069055204851e-05, P153=9.371232474482952e-05, P154=8.206315000554782e-05, P155=0.00010439852958129619, P156=0.00013041430520055545, P157=0.00011281471617704102, P158=0.00015319657196053043, P159=0.00014907125556089642, P160=8.343602610361397e-05, P161=5.829158109774053e-05, P162=4.983773229197582e-05, P163=3.8738637877939164e-05, P164=6.936819868826772e-05, P165=6.183101170967835e-05, P166=6.459079846876198e-05, P167=5.1188339624642806e-05, P168=4.415277778946506e-05, P169=6.061476453406953e-05, P170=4.481107292755905e-05, P171=4.6184877911902e-05, P172=4.158166770637054e-05, P173=4.066599970350407e-05, P174=3.881634845642896e-05, P175=3.819266488514586e-05, P176=2.7728596524749105e-05, P177=1.7350823792282526e-05, P178=6.600943839369896e-06, P179=8.579536857860291e-07),Row( P0=-592.6091256189156, P1=46.989091174672005, P2=-11.307107720782778, P3=-1.1884813372377991, P4=-13.112915815927765, P5=-10.78500103660663, P6=-9.343054254940572, P7=-4.741778416508287, P8=-3.2303376556495507, P9=-2.65926147224465, P10=-9.831958433749474, P11=-7.103138691092707, P12=-4.649450174619417, P13=-8.969743470429584, P14=-5.08408750438981, P15=-3.1553174362830165, P16=-3.1413153808329004, P17=-2.5424638318329094, P18=-2.0747572400135086, P19=3.908084173836281, P20=0.3928882792755594, P21=5.153547119632912, P22=2.480607900959571, P23=3.242977509680621, P24=1.5857132153908031, P25=1.737240972486833, P26=-1.048404456587465, P27=-0.30864952180363625, P28=-0.8651267544143907, P29=-1.3924117329921306, P30=0.9655449301566761, P31=1.9632476857324652, P32=-1.8400584321208744, P33=0.5631959940255696, P34=-2.481076243716242, P35=0.13192324496688967, P36=-0.7414736308548581, P37=-1.7506026917526545, P38=-1.9439674337794262, P39=-0.6260175077248412, P40=0.6948015323631689, P41=0.6541089085627894, P42=0.6636985478641779, P43=0.670050866421241, P44=0.6981418211593727, P45=0.7358825303849165, P46=0.819058205222075, P47=0.7487511932728868, P48=0.6812407794831504, P49=0.6801290719136259, P50=0.6905839512269829, P51=0.7054294080606187, P52=1.3390538076926521e-05, P53=2.0975735108476578e-05, P54=3.6650013729217744e-05, P55=5.6065424638750276e-05, P56=9.167463691417874e-05, P57=0.00027464561479264513, P58=0.0031972474401223533, P59=0.011500097968512063, P60=0.013806404763061587, P61=0.03928981417951906, P62=0.1834474292610707, P63=0.31485157902653654, P64=0.11342630358769314, P65=0.04605050833897333, P66=0.05000806909618756, P67=0.035874791168477214, P68=0.009189223548929041, P69=0.0060602289148184605, P70=0.007424626714898686, P71=0.00880473966596097, P72=0.017537068381147717, P73=0.02057833789909817, P74=0.03441771701267894, P75=0.06737630156619018, P76=0.06511022446785114, P77=0.05486798291195902, P78=0.05732560947621493, P79=0.06643701608389967, P80=0.07452515054660772, P81=0.11256491683292377, P82=0.2985637450843535, P83=0.2126991696379599, P84=0.04828878478328503, P85=0.02377313407539922, P86=0.03581010436211908, P87=0.03408913149291514, P88=0.01983071569496652, P89=0.0119954066221838, P90=0.016023270731438825, P91=0.02744268556729225, P92=0.06961754667746833, P93=0.056929643748411037, P94=0.07402929499756324, P95=0.07094424760752964, P96=0.013128603271429928, P97=0.0021355686904668504, P98=0.008158195974351802, P99=0.04654784721313448, P100=0.029617098025504923, P101=0.018492204259789643, P102=0.031118091005400278, P103=0.014898300619287408, P104=0.014851224172981991, P105=0.02618931512775843, P106=0.02279951015565132, P107=0.027721275412889518, P108=0.016602447598191503, P109=0.040922230434558816, P110=0.04038425740436795, P111=0.012780791952376743, P112=0.009002294026819664, P113=0.0050258350462102684, P114=0.002198797735821197, P115=0.001953878159701499, P116=0.0022136082754314382, P117=0.0013312047777008398, P118=0.0008586908302809587, P119=0.0010483921847879655, P120=0.0007897044089431197, P121=0.000991274128021058, P122=0.0011037530059489224, P123=0.0010339987639039125, P124=0.0006791064897482755, P125=0.001047556732592063, P126=0.0004870006000696889, P127=0.0013725572377217708, P128=0.0008012482071242912, P129=0.0015124210507988338, P130=0.0006910942671710384, P131=0.0007703955308715093, P132=0.0009698562180815734, P133=0.0010726824967524204, P134=0.0012324286947523317, P135=0.001432990204836655, P136=0.0023544912275002073, P137=0.0021911726435399205, P138=0.001714423171036101, P139=0.0030781550102199663, P140=0.0025973948426509896, P141=0.0007489000235606823, P142=0.0007451981564324722, P143=0.0006048945493659875, P144=0.0007038470986861096, P145=0.0010644100145195184, P146=0.0009439548137660464, P147=0.0005119133255500288, P148=0.0006045341390738269, P149=0.0004028001315380892, P150=0.0004506405777825223, P151=0.000997815351106008, P152=0.0012507069266978, P153=0.0008297019886686458, P154=0.0007024488089857866, P155=0.0005260680993387745, P156=0.00046291667543070807, P157=0.000482049021831315, P158=0.00042816640269963824, P159=0.0008951710962776265, P160=0.000616715677170133, P161=0.0005774736618401716, P162=0.0006316072709643279, P163=0.0003970706751341496, P164=0.00031502322526811264, P165=0.0003793741877893869, P166=0.0005352313610468351, P167=0.00025157684119868806, P168=0.00025159339281937516, P169=0.0002295532809902838, P170=0.00023869112026453249, P171=0.0002861528784050944, P172=0.00018312232913703357, P173=0.00014376567942560434, P174=0.00025932811464342244, P175=0.0002857865715408915, P176=0.0002457023656355395, P177=0.00013079676278863597, P178=6.431696844283111e-05, P179=8.422297809642957e-06),Row( P0=-584.8989994223925, P1=40.16455362401104, P2=-10.35768818997365, P3=-0.9272457223246036, P4=-8.863333964493721, P5=-9.060995345910287, P6=-8.899583179063308, P7=-5.63251970525104, P8=-5.320861493443133, P9=-3.3767501875632933, P10=-9.467133562850046, P11=-7.034543215041918, P12=-3.5377049835474725, P13=-6.885528631556425, P14=-4.475543058327796, P15=-3.3439831314012083, P16=-3.033252974737387, P17=-0.019557278803833258, P18=-1.0316955244951869, P19=4.109310179719012, P20=1.204815330941216, P21=4.932213028728293, P22=1.9853563333983886, P23=2.254283471770248, P24=-0.7740049285080772, P25=0.5692111318283901, P26=-1.7983771006084728, P27=0.32795797718182507, P28=-0.6189976722702927, P29=-1.1756549956112576, P30=-0.03971072856074558, P31=0.9000048563391744, P32=-0.6568443780270177, P33=1.5121014037802942, P34=-0.5116069752932979, P35=-0.7164449898309341, P36=0.5691060424391639, P37=0.0879556243299108, P38=0.5284557846664871, P39=0.08061147878305713, P40=0.6924993673578281, P41=0.6563681151650245, P42=0.6446219570607254, P43=0.6503585650413501, P44=0.7063994823965397, P45=0.7510222843453516, P46=0.749017886847438, P47=0.6847958683405756, P48=0.6678941655584857, P49=0.6680695658014962, P50=0.6806539374852055, P51=0.6745530991262613, P52=6.070543255344078e-06, P53=1.6432554897558696e-05, P54=2.4043605344682794e-05, P55=4.875181637853567e-05, P56=0.00010572008135614925, P57=0.0005066427573421528, P58=0.0031248383215337415, P59=0.009403995354202816, P60=0.011622638277595571, P61=0.02826091968603509, P62=0.16430575161339273, P63=0.2244190999815779, P64=0.1728224240235609, P65=0.1610730085180138, P66=0.08225407416594116, P67=0.08912110148962295, P68=0.046338327260546534, P69=0.01181655601880978, P70=0.012843740241263587, P71=0.009370713931161805, P72=0.009403065252992403, P73=0.0346534821850428, P74=0.08529517481258315, P75=0.05311833348922545, P76=0.03448465615070961, P77=0.1661223998703073, P78=0.4630765217064729, P79=0.5230809754220792, P80=0.2153276684766132, P81=0.036959318181905905, P82=0.0280810369995531, P83=0.03167326743277941, P84=0.06959382755814278, P85=0.10097979586014363, P86=0.0619610989646787, P87=0.04174093680635143, P88=0.03095807128031244, P89=0.0277255343811583, P90=0.05091185098665856, P91=0.035479995252927714, P92=0.01811225953604553, P93=0.011368968229795641, P94=0.030290713765394137, P95=0.07550562017444053, P96=0.06182504191658708, P97=0.0736269182900885, P98=0.06368817835813108, P99=0.043968261918234545, P100=0.017585578979781465, P101=0.01704191711512378, P102=0.02518906172976855, P103=0.01162574030302639, P104=0.006157851408418938, P105=0.009040201584672084, P106=0.014219292937745959, P107=0.031231533422321697, P108=0.06123886180636799, P109=0.07521734287453255, P110=0.10168131481903747, P111=0.0613190413420637, P112=0.03218209071667969, P113=0.008646541308849174, P114=0.005349907841770084, P115=0.005578331835296721, P116=0.008222848531205654, P117=0.0022584149580695445, P118=0.0019335929285824146, P119=0.0020635247243609434, P120=0.0015126394200125415, P121=0.0013980578937493624, P122=0.0015037163191834728, P123=0.0006421966408891728, P124=0.0014133652901034644, P125=0.0016125987908203346, P126=0.0010967642799098738, P127=0.005305062422786394, P128=0.009850505427800289, P129=0.0036522092947850835, P130=0.004662526621022295, P131=0.001295347154924154, P132=0.001729012004957775, P133=0.0026832430023990255, P134=0.002407967506783636, P135=0.0039976532950947614, P136=0.004752910482272447, P137=0.0071174065618028245, P138=0.007170297815610155, P139=0.005019010051890902, P140=0.0042958333569287485, P141=0.0025838331120275975, P142=0.00223546916332479, P143=0.003009018780937602, P144=0.0025976498393618275, P145=0.002378470168219265, P146=0.0007426130679327715, P147=0.0003579468004795331, P148=0.00040978095964123247, P149=0.00037666178418520807, P150=0.0006244828637711887, P151=0.0010957208687551493, P152=0.000959607499469187, P153=0.0010500002467120453, P154=0.0010667269351633969, P155=0.0010635445714269524, P156=0.0006273255934580976, P157=0.00043471398377573203, P158=0.0005964284712771842, P159=0.000789808970647215, P160=0.0007273382697734716, P161=0.0006623030859037607, P162=0.0005268786551621459, P163=0.0004726017706483755, P164=0.00034577671809338494, P165=0.0002530000213934638, P166=0.00023993278415152176, P167=0.0002136748978741298, P168=0.0002597015487014712, P169=0.0002560674443923591, P170=0.00021826714297255276, P171=0.0002669468573840592, P172=0.0003301405162999519, P173=0.0002680418874021764, P174=0.0003139724619937073, P175=0.0002010873679092315, P176=0.0001074485131587553, P177=9.257435430116044e-05, P178=5.895490579366312e-05, P179=8.741209884045797e-06),Row( P0=-674.4905973130683, P1=62.58134722243844, P2=-9.308620430969471, P3=2.055364500854928, P4=-6.2366771624248765, P5=-12.593002870685455, P6=-5.947880792253987, P7=-2.7742269363375347, P8=-4.900494597201729, P9=-4.380557092529139, P10=-7.201330301704658, P11=-6.1945463540382635, P12=-2.195722884594912, P13=-9.708597797358836, P14=-5.682459652380655, P15=-4.645880396798899, P16=-4.3686235823548625, P17=-3.9060774612499363, P18=-5.872633734621591, P19=-0.9840615819426121, P20=-6.230571842470602, P21=0.025412753801325568, P22=1.0949566339077004, P23=4.634253855168134, P24=4.203768806578068, P25=4.728982266605954, P26=2.5727638307784946, P27=2.4998607826770463, P28=1.8019240230000502, P29=2.4605566047161997, P30=-0.6911384028964405, P31=2.713969503414348, P32=-1.290293067703464, P33=2.8548339775585383, P34=1.1468711835695629, P35=0.5083195935832422, P36=-0.025471783531331133, P37=2.3015243329199206, P38=-1.361057767421933, P39=0.04464628692299669, P40=0.6861673172417705, P41=0.657700609037487, P42=0.6446134916719782, P43=0.6426799719572571, P44=0.6977038350436688, P45=0.7659283818560085, P46=0.7586730746316883, P47=0.6719342112391165, P48=0.6353343859330258, P49=0.6602110774177602, P50=0.6684361143323645, P51=0.6704113147705276, P52=2.62416398879331e-06, P53=9.776243876839253e-06, P54=7.3121101613401224e-06, P55=6.051979026908576e-06, P56=2.704739291047403e-05, P57=0.0010930225339644338, P58=0.00997336984370582, P59=0.01821458561820449, P60=0.02444011401232459, P61=0.05284155783092496, P62=0.07351352201819335, P63=0.035748923563640296, P64=0.004145515720803529, P65=0.0008355922864674079, P66=0.0022863957536067513, P67=0.003128695077642597, P68=0.00318568928550903, P69=0.005803566999345828, P70=0.013063538159404697, P71=0.014889533303948559, P72=0.022597685231857536, P73=0.02213855295685321, P74=0.017695048539266067, P75=0.01596245681959453, P76=0.004668863612351245, P77=0.002743820349619006, P78=0.003647409073882585, P79=0.012110523097718873, P80=0.03269282431622133, P81=0.023053904907395163, P82=0.01321413537417734, P83=0.015023511779667173, P84=0.017606764546968415, P85=0.010760892190307072, P86=0.008961142545473784, P87=0.006652669606330787, P88=0.00256358111159658, P89=0.001963761405852665, P90=0.0024575057520819384, P91=0.0054838201642978925, P92=0.007947603394067235, P93=0.005545395372304083, P94=0.008274718272714731, P95=0.005181263490997957, P96=0.00236450318809277, P97=0.003133109754692805, P98=0.009484488731686154, P99=0.009252959299043201, P100=0.0014146637819187336, P101=0.00015644093388265414, P102=0.0002565843742078053, P103=0.000739666835883027, P104=0.0009382943676390162, P105=0.0006310719626999137, P106=0.0006560420132356154, P107=0.0002612679064436772, P108=0.0008046626051987779, P109=0.0025476644062546834, P110=0.0007557266786255816, P111=0.00037281545173347205, P112=0.0008794211583688356, P113=0.0025865735051127663, P114=0.0013933817452845064, P115=0.000943375468652378, P116=0.0009320903681207381, P117=0.00047780423328837426, P118=0.0003955384500321192, P119=0.0005665937925733669, P120=0.0010315795447544789, P121=0.0007882186572839397, P122=0.0005527398695195804, P123=0.00036069623348071754, P124=0.0001921538622349985, P125=0.0002878338797992953, P126=0.00037182265004776004, P127=0.0005111976723713746, P128=0.00048480254998040675, P129=0.0005315744186347738, P130=0.00013943706673317255, P131=0.00014714530709429327, P132=0.0002539403675915975, P133=0.00030713921561974065, P134=0.0003461461499980115, P135=0.000369001899655708, P136=0.00017827020144971177, P137=8.678527495815404e-05, P138=7.594271160575232e-05, P139=3.3297480282698164e-05, P140=3.816865021321722e-05, P141=3.8088636671320657e-05, P142=5.1103234928670005e-05, P143=6.117721629541585e-05, P144=4.37431784997858e-05, P145=5.188165176449486e-05, P146=2.077948334861599e-05, P147=4.793018298223518e-05, P148=1.6498720953948312e-05, P149=1.9847778598109856e-05, P150=4.7900879693000185e-05, P151=5.6047983731557965e-05, P152=3.644895402906914e-05, P153=2.9165672160005782e-05, P154=4.98772663978856e-05, P155=4.357948569776852e-05, P156=2.8637183678773323e-05, P157=3.7297438146903706e-05, P158=3.654477423388832e-05, P159=3.5406403707586525e-05, P160=2.8361037928062524e-05, P161=2.5171854800074355e-05, P162=2.1167336921358972e-05, P163=2.706826833189554e-05, P164=2.3971894599320127e-05, P165=1.764915048515048e-05, P166=2.703745925732942e-05, P167=1.3908709339995673e-05, P168=2.8933967498439283e-05, P169=1.610672532271457e-05, P170=1.1048563366740001e-05, P171=9.347058801379178e-06, P172=8.288520042037647e-06, P173=1.3195340235812482e-05, P174=9.476412780508046e-06, P175=1.1138691202321837e-05, P176=1.1265094313896795e-05, P177=9.694318373705038e-06, P178=5.288206856455824e-06, P179=4.030645580482122e-07),Row( P0=-660.1598606450439, P1=43.03753412007063, P2=-18.631252961220834, P3=-1.8665250555953188, P4=-12.097869165144788, P5=-17.70374017669945, P6=-8.111096248825548, P7=-7.040818999478031, P8=-5.152360051648807, P9=-6.016855792242407, P10=-12.129762504961072, P11=-7.212873817242488, P12=-4.718584000310755, P13=-4.251505528543627, P14=-0.32834837496341085, P15=2.774543900727574, P16=4.8581556197988265, P17=4.249137099290202, P18=-1.4395828765100889, P19=4.54195387231643, P20=-0.6243423629995232, P21=1.1150955730233407, P22=1.6671053406353016, P23=2.49608196655903, P24=-1.4834567294590042, P25=1.6419444502132718, P26=-4.03626839958352, P27=-0.23630376784935012, P28=-1.4450420988767658, P29=-0.18773940687936314, P30=-1.2936460982847784, P31=1.4276278207215645, P32=-3.8981092595959757, P33=-1.0942322755888703, P34=-3.4830378219193183, P35=-0.7106763587366282, P36=0.08715077649847178, P37=-1.3665544372052476, P38=-2.5336485051924424, P39=-0.9719378102355576, P40=0.7149318768899569, P41=0.7134656429237578, P42=0.7143987258114409, P43=0.6962121508798867, P44=0.66526520125653, P45=0.6591210642056353, P46=0.6949069901979574, P47=0.694758488197771, P48=0.6488535595915113, P49=0.678626330822515, P50=0.7178912887441989, P51=0.6953058876913167, P52=9.324951889748408e-07, P53=3.3299245474424475e-06, P54=8.120124840818693e-06, P55=2.498791124169518e-05, P56=4.147159983725217e-05, P57=4.348602210497646e-05, P58=9.068837995088245e-05, P59=0.00032381342099944505, P60=0.0008520140981948144, P61=0.003504134699132059, P62=0.043337864840688536, P63=0.0745684347205487, P64=0.031552425516803054, P65=0.0500695249794491, P66=0.07477113288621395, P67=0.05845708169688267, P68=0.04499053091092563, P69=0.044710654742423604, P70=0.05527121303373988, P71=0.02613230662179871, P72=0.0034618833286121993, P73=0.005822940288442138, P74=0.022103697253900708, P75=0.03564455719929874, P76=0.022330714316017828, P77=0.012978985883692961, P78=0.020701119244906155, P79=0.04458687260237207, P80=0.05914796001917607, P81=0.049533580311096385, P82=0.05639878327974016, P83=0.06966561858239427, P84=0.11835750537372197, P85=0.09659216596752158, P86=0.07287642137046198, P87=0.09515379115119832, P88=0.09472223942947432, P89=0.059656544263357834, P90=0.03823995897887961, P91=0.008020837688356357, P92=0.00524912696741962, P93=0.006066027108030111, P94=0.014795661716233829, P95=0.020565653501210777, P96=0.008715918030538736, P97=0.012133538632019384, P98=0.015336618341807535, P99=0.01995487479674007, P100=0.03376073095091646, P101=0.020785064251523343, P102=0.012351451504689744, P103=0.00231799018962998, P104=0.0025065970433880563, P105=0.009074495187489243, P106=0.010557943433790592, P107=0.0029221236107488504, P108=0.002188172953928217, P109=0.0025563528235932657, P110=0.006138175008249352, P111=0.013131959439713686, P112=0.0026199977026708557, P113=0.0038873874271352876, P114=0.009461685239628018, P115=0.010838097953500984, P116=0.019572392388091817, P117=0.013242948623019312, P118=0.002969914781991453, P119=0.0026025949167865783, P120=0.0033115068411472726, P121=0.001312402001485093, P122=0.000568247853608717, P123=0.0006845937270715074, P124=0.00034221415792876466, P125=0.0005461078502162775, P126=0.0006049386194340779, P127=0.0006175565461633045, P128=0.0005320142904634762, P129=0.0007304295992584558, P130=0.0007705952377243881, P131=0.0013676571734939644, P132=0.0011123365033594089, P133=0.0008665778622554562, P134=0.0010527930829586037, P135=0.0007712002221835378, P136=0.0005925585335993098, P137=0.00030571122909432377, P138=0.00020111346537113204, P139=0.0001696029937757056, P140=0.00016960282522709717, P141=0.00016025410131353113, P142=0.0002722547565610482, P143=0.00041966537421546753, P144=0.0005645897486663497, P145=0.00033433675976725515, P146=0.00013824851834149117, P147=5.798122345579804e-05, P148=3.897420190195904e-05, P149=2.7313572414815184e-05, P150=3.99788707112692e-05, P151=5.2468327634303544e-05, P152=6.631744974124843e-05, P153=0.00011482633162312515, P154=8.007274504654069e-05, P155=5.9425780467141836e-05, P156=8.498257221687913e-05, P157=4.034641743767244e-05, P158=5.054432715204072e-05, P159=4.381116248106737e-05, P160=3.802040291760636e-05, P161=4.0974904174522225e-05, P162=2.8848614404108816e-05, P163=2.1266563308433864e-05, P164=1.9683975199904676e-05, P165=1.1357263737831474e-05, P166=2.3462422338264553e-05, P167=2.3862270759610265e-05, P168=3.162928679784608e-05, P169=3.605682806143874e-05, P170=3.5731370806311065e-05, P171=1.32011035778942e-05, P172=1.2332070034886607e-05, P173=3.90817844240456e-05, P174=2.979240138052041e-05, P175=2.6516544688311217e-05, P176=1.2150375866126886e-05, P177=8.66851482710479e-06, P178=4.008182241516728e-06, P179=3.5574189719197444e-07),Row( P0=-681.3929922903607, P1=39.319010381773744, P2=-12.608030492632668, P3=0.5915709580480019, P4=-9.360460793185167, P5=-13.694260234849759, P6=-8.821184791135645, P7=-5.677290657688257, P8=-3.022140237653569, P9=-5.327020445141567, P10=-8.977141489983175, P11=-6.565461233481698, P12=-5.083407301368188, P13=-7.124806751749037, P14=-4.614994685032838, P15=-5.178366755687365, P16=-5.1417524438089055, P17=-3.755541901611373, P18=-4.709243319131156, P19=-2.4040620285437373, P20=-5.23201947848198, P21=-1.216058824156247, P22=0.5171402318439569, P23=1.6596063715320957, P24=1.7673556253780496, P25=5.539449609009709, P26=4.938817832660776, P27=2.979754773315152, P28=4.673496639593526, P29=3.685119830221152, P30=0.9315280929918117, P31=1.905527528481594, P32=-2.4974781062989386, P33=-1.3967572134026474, P34=-1.2566530741134465, P35=0.07606477959362212, P36=-0.5022315066775924, P37=-0.6985833307729085, P38=-3.005896564788197, P39=-0.22199560922094094, P40=0.6078389563493946, P41=0.6395302771743698, P42=0.6936647293996497, P43=0.7483943527442117, P44=0.7095299780447655, P45=0.6938408074761485, P46=0.6322587489084945, P47=0.5584816539080751, P48=0.5438815916227582, P49=0.5749695439357747, P50=0.6090597161943629, P51=0.578480547702617, P52=5.975099461099391e-07, P53=2.106098966362368e-06, P54=8.773779516716698e-06, P55=2.8102177194826736e-05, P56=4.657041527113318e-05, P57=9.489651467834391e-05, P58=0.00040256975239446376, P59=0.004986116192594416, P60=0.03737089597877398, P61=0.07101079005062357, P62=0.08136150195306745, P63=0.05951423347031846, P64=0.010466477210195354, P65=0.001460295826408071, P66=0.0009415357146651233, P67=0.0011549653541834543, P68=0.003694936732265494, P69=0.012803579103651389, P70=0.02956613075128979, P71=0.03820041649855405, P72=0.02053726205062842, P73=0.018047808700581033, P74=0.022950238406177343, P75=0.018415558982999156, P76=0.010935882558122938, P77=0.006428165449817111, P78=0.004477636718051758, P79=0.006711596157181796, P80=0.018697980441845768, P81=0.02272958536784411, P82=0.014958243175845378, P83=0.010037682410383594, P84=0.014165519416706981, P85=0.017291783171509208, P86=0.018520031744787203, P87=0.024321399914108224, P88=0.016171930046406498, P89=0.013007847097588156, P90=0.010850906686580387, P91=0.0018351315807972038, P92=0.002119005536580479, P93=0.04050412257160505, P94=0.09158638816288253, P95=0.032908280009469076, P96=0.007862183646696072, P97=0.002096460461814361, P98=0.00047941103436110864, P99=0.0003743073791252529, P100=0.001439706975627981, P101=0.0010802463009196741, P102=0.0007282965306155694, P103=0.0004111964688753581, P104=0.0002081307606182722, P105=0.0009664126682641263, P106=0.006336741610721604, P107=0.0030543074060550564, P108=0.001472321594414457, P109=0.0012450505290533477, P110=0.010542684967462673, P111=0.023062472309461075, P112=0.005521392105533555, P113=0.0030447513522044176, P114=0.006565951202068565, P115=0.0061548146751526905, P116=0.0013179716767122648, P117=0.0009979080313296598, P118=0.0011838195922324573, P119=0.0010700311686815396, P120=0.001027568194479395, P121=0.000626189223759845, P122=0.0013191182177452383, P123=0.0012101862307138398, P124=0.0005369632851505517, P125=0.00042845260108814986, P126=0.00016510986518928586, P127=0.0002206354873829645, P128=0.00017893799836388996, P129=0.0001576765142222334, P130=0.00023132048183986936, P131=0.00034383479616912705, P132=0.0004650817196405679, P133=0.00040977910162013346, P134=0.0004439132220341118, P135=0.0005367581620164446, P136=0.0004269645938568, P137=0.00017514304708416835, P138=0.00012325954245878088, P139=8.414877882497583e-05, P140=0.00010028348296166217, P141=0.00012896995791133233, P142=0.0001841859642794898, P143=0.0001387378296929991, P144=0.00013363846567966754, P145=0.00013088822184935655, P146=7.48979003417016e-05, P147=4.4522810222635435e-05, P148=4.863206851400283e-05, P149=7.320390402763766e-05, P150=0.00011525751925145581, P151=0.0002147155406299546, P152=0.0003184309487103792, P153=0.0002473417315813618, P154=0.00030239007426655025, P155=0.000209441164270113, P156=0.00043389648015864144, P157=0.00030603024749258176, P158=0.0002605281363147965, P159=0.00022201216299592205, P160=0.00020698868330690077, P161=0.0001401157395317329, P162=0.00015827717966618359, P163=0.0001114043823064603, P164=7.431477528829733e-05, P165=5.1957129859121534e-05, P166=4.374183145082717e-05, P167=4.092051135960393e-05, P168=7.5634368831291e-05, P169=6.039906041372133e-05, P170=3.90628120784582e-05, P171=2.914635153259021e-05, P172=3.333275222205183e-05, P173=5.465855335062269e-05, P174=4.729030435917791e-05, P175=4.9347817403077645e-05, P176=2.39086461500966e-05, P177=2.6230410332503904e-05, P178=1.8141932936632703e-05, P179=3.006393933214033e-06),Row( P0=-554.0646708525732, P1=36.05745194481212, P2=-22.835549719474304, P3=-1.1442753694578227, P4=-15.25691513801516, P5=-20.402680066502583, P6=-6.330063712592046, P7=-5.373345574732376, P8=-4.920395158240536, P9=-5.516970554973681, P10=-11.032976294896352, P11=-6.078766111789772, P12=-6.1341820052930744, P13=-10.128439309559848, P14=-4.280476057850594, P15=-5.421137081665062, P16=-3.6337212491190627, P17=3.3469126218902607, P18=0.7281320537691766, P19=11.222477628118401, P20=6.254869333253726, P21=6.735228823290712, P22=4.963585571496294, P23=1.661975394951777, P24=-1.2975354373615935, P25=1.7570372973645505, P26=-1.3736552172242424, P27=-0.12409031400147012, P28=-1.7291727532498946, P29=-0.6937197769041196, P30=-3.881494587003622, P31=-0.14833042512027148, P32=-4.636403139460206, P33=-2.1879512576713287, P34=-4.027724169741542, P35=-1.3699754309681018, P36=0.7415523450884535, P37=-0.4411404342529331, P38=-0.18035143411584534, P39=0.1887493534246405, P40=0.5689694293372844, P41=0.5859767974650419, P42=0.6300068921953145, P43=0.6073778135648874, P44=0.6341598952988314, P45=0.6131270957057203, P46=0.6645747075335994, P47=0.7030778637636856, P48=0.6744316023886772, P49=0.7111765816101783, P50=0.6814748497284154, P51=0.6188037544873198, P52=0.00014292304831999583, P53=4.254195623188003e-05, P54=1.666012784684265e-05, P55=3.505675924030689e-05, P56=4.354805300300327e-05, P57=3.9999171678531025e-05, P58=4.350546213997537e-05, P59=0.0005757123838762428, P60=0.008806156912383067, P61=0.030241556650351366, P62=0.1074632675941225, P63=0.20054383601344491, P64=0.1895811513235975, P65=0.394322443089817, P66=0.2845644086235376, P67=0.03453307842590655, P68=0.005870529563855027, P69=0.0043958062566861935, P70=0.0063375041485374045, P71=0.012381305248587464, P72=0.01701760126625309, P73=0.020437775380617553, P74=0.0448509023392625, P75=0.08143698230345511, P76=0.10893366663826617, P77=0.17176443242570255, P78=0.31801242652182915, P79=0.8650031020022013, P80=1.6204775819060193, P81=0.5841701902992772, P82=0.06766894255944733, P83=0.023914502441595854, P84=0.03329144367358436, P85=0.029836118076280352, P86=0.05251997414659664, P87=0.0696709898139314, P88=0.08704086295397072, P89=0.059161063397810895, P90=0.08958613389755028, P91=0.4445691315259838, P92=0.5094846545542852, P93=0.03685555345078339, P94=0.02620332572799181, P95=0.01673911311939463, P96=0.06894928026398031, P97=0.2520092150679596, P98=0.06142724567504029, P99=0.040644919527433127, P100=0.057049026918058236, P101=0.043846311709324695, P102=0.009494991773851453, P103=0.032804533545187246, P104=0.03736216583841272, P105=0.003513101114490133, P106=0.0032473358049409816, P107=0.02057970274872688, P108=0.016078977152614908, P109=0.13461567834718677, P110=0.08894367885497839, P111=0.0014212954650735903, P112=0.00854581386406707, P113=0.07350286590510269, P114=0.033051285717202374, P115=0.0031294105627354403, P116=0.005449402017172587, P117=0.00633203152909547, P118=0.01393340136937772, P119=0.00552320214581803, P120=0.0028510223598479075, P121=0.0065293144788097935, P122=0.01674896294557821, P123=0.004484470767991651, P124=0.0025048934027969033, P125=0.0058307887943086865, P126=0.0064361322226847975, P127=0.0029049704601984824, P128=0.0023149591617510463, P129=0.012663647952282364, P130=0.005454579699312411, P131=0.002680423823762231, P132=0.005895890793347365, P133=0.0035120118202098772, P134=0.0031034424027641693, P135=0.006527990697564372, P136=0.0014239678474059186, P137=0.0014716320098675224, P138=0.001067302840846715, P139=0.0010664930686318562, P140=0.0011506720672484635, P141=0.0015230188772917528, P142=0.0017131266293755425, P143=0.0021696737661925595, P144=0.0027406534536518036, P145=0.0012826317544537165, P146=0.0007002654831192592, P147=0.00010572969112440006, P148=9.386663349172613e-05, P149=0.00010311817111104429, P150=9.582676993642198e-05, P151=7.451783544157626e-05, P152=9.864850692822862e-05, P153=0.00012664470950921104, P154=0.0001296762258583743, P155=0.00012331341761347893, P156=0.00013525463407700587, P157=0.00012302607953628583, P158=0.00020909168793969334, P159=0.00023170354825553184, P160=0.00012388057126853984, P161=8.075568279474175e-05, P162=5.0941001314133764e-05, P163=5.903383248275277e-05, P164=6.000802014715821e-05, P165=6.716110764344562e-05, P166=5.510619912345116e-05, P167=5.80801232125938e-05, P168=9.797292631883055e-05, P169=9.269933020874843e-05, P170=0.00010146084068506453, P171=7.388061839474911e-05, P172=6.352020208513721e-05, P173=6.170890885317742e-05, P174=5.93940438334059e-05, P175=8.42251652975721e-05, P176=5.1922161498328005e-05, P177=3.923092729743215e-05, P178=1.9754017079491248e-05, P179=2.7364171456141997e-06),Row( P0=-654.1687544849808, P1=37.35469995305175, P2=-10.926316464890007, P3=2.9600837356905263, P4=-8.078842919515925, P5=-13.218290177229147, P6=-7.491045082129275, P7=-4.465271477075936, P8=-2.6186712552242364, P9=-4.438641761183459, P10=-8.705671901441429, P11=-5.727077409452967, P12=-2.7420603758593454, P13=-6.999086088626509, P14=-2.1689712771819822, P15=-3.2937717472781185, P16=-3.195871870723024, P17=-2.2537600193589458, P18=-4.502572919787583, P19=-1.1038919121694037, P20=-4.390570841138319, P21=-0.20807041382366892, P22=1.4608831495762749, P23=0.6428952714392551, P24=1.2522553643914387, P25=5.305677339992084, P26=4.828202686302758, P27=3.01317864131837, P28=6.3134484305368845, P29=4.569975210505964, P30=2.0786667874491576, P31=3.143697373814925, P32=-1.8813742167190677, P33=-0.9198505102641023, P34=-0.6203267902769649, P35=-0.06246548450063116, P36=0.05426968697091549, P37=-0.41109789330571317, P38=-3.3597759494955257, P39=-0.45730623854636826, P40=0.6519691222963189, P41=0.7018665339741741, P42=0.7359225810531733, P43=0.7872127238936022, P44=0.7410260216468577, P45=0.6983197346016656, P46=0.6935480370375016, P47=0.645523475988744, P48=0.630310770637446, P49=0.6458766496586165, P50=0.6588082353171626, P51=0.6223736830311689, P52=2.511085873289206e-06, P53=9.653131571855328e-06, P54=9.540032775786026e-06, P55=8.544600274509015e-06, P56=1.8893991723358806e-05, P57=4.953607350599319e-05, P58=0.0002640297787458445, P59=0.006440951664931697, P60=0.03722033661254925, P61=0.04839672696846196, P62=0.08267255608035888, P63=0.0971372896134359, P64=0.01062040800977925, P65=0.0011510713037544786, P66=0.0010787960275722487, P67=0.0023292787257592883, P68=0.009684902296976352, P69=0.027398350079106848, P70=0.04419191217611977, P71=0.029151412647608458, P72=0.010913366135478508, P73=0.009925258052052395, P74=0.02658547337069188, P75=0.019016833013040423, P76=0.0058940907431481015, P77=0.005278233981735136, P78=0.006718967886338647, P79=0.010021932446288469, P80=0.022045059071594104, P81=0.019827067131049893, P82=0.011037853820678668, P83=0.008604381577237377, P84=0.012191774798025383, P85=0.01863075700715186, P86=0.023218481996851458, P87=0.01588900560778378, P88=0.009577236856310498, P89=0.020022525277841808, P90=0.017606475280043374, P91=0.0017255924894204115, P92=0.0017124316921250428, P93=0.0229337124605868, P94=0.23149167475074972, P95=0.10590286461803468, P96=0.005513088860532206, P97=0.00037779451773167446, P98=0.00031050625465669466, P99=0.00016577010326519282, P100=0.0010814611092028797, P101=0.002069826179072786, P102=0.0009693290207372497, P103=0.0002018512050101359, P104=5.987969997227532e-05, P105=0.00030779945927335875, P106=0.007942154935535255, P107=0.007209753095926281, P108=0.0012523358963373293, P109=0.00022086214121268418, P110=0.005797539752385321, P111=0.03268801604415485, P112=0.023360003070564364, P113=0.011450485431377859, P114=0.003007005064689444, P115=0.010407465185970877, P116=0.0032258074465912273, P117=0.0020843472863607553, P118=0.000660990223234664, P119=0.0008959916669207435, P120=0.0012824899482766488, P121=0.0009671906181340238, P122=0.0010451649459096311, P123=0.001049925565609248, P124=0.0007540601012752085, P125=0.0007160397415456991, P126=0.0003793096399246808, P127=8.838728339038071e-05, P128=0.00017163391050064564, P129=0.0002393300831995791, P130=0.00022890660021608708, P131=0.0004768872949387254, P132=0.0006822431812469461, P133=0.0009824699708320516, P134=0.0016604010098366887, P135=0.0011755815135049172, P136=0.0007899310316928314, P137=0.00035992117718086186, P138=0.00027157239157168814, P139=0.00013857570339078523, P140=0.00021737070419657964, P141=0.00026546606006005194, P142=0.0005916709089351851, P143=0.00063358120575373, P144=0.0003073713422686788, P145=0.00019456577963176296, P146=8.875920877537587e-05, P147=4.747614527352658e-05, P148=5.298403953267883e-05, P149=8.018746898908561e-05, P150=0.00011340909300954073, P151=0.00011346049949343754, P152=0.00012959143385612152, P153=9.984427128423428e-05, P154=0.000181382044549917, P155=0.0002023960826442609, P156=0.0001862684926188189, P157=0.0002414509374315594, P158=0.00026225670960214684, P159=0.00034956676130853763, P160=0.0004489204958366051, P161=0.0002833175683619073, P162=0.00020022439975594535, P163=0.00014129523993548654, P164=0.00012175827809689291, P165=7.17710453095683e-05, P166=5.8877936630633586e-05, P167=8.340982788036711e-05, P168=0.0001273422850519279, P169=0.00012335593620643596, P170=8.843964591268442e-05, P171=5.648695462672228e-05, P172=6.729899838258684e-05, P173=5.714107391731235e-05, P174=4.3361771751292674e-05, P175=4.7095405950524276e-05, P176=4.5329884381683316e-05, P177=2.8173456089316667e-05, P178=1.5812995680060804e-05, P179=2.1690027742775494e-06),Row( P0=-604.7265498893688, P1=38.61877677809204, P2=-17.622290974462512, P3=-3.0931648980710387, P4=-10.032430277074978, P5=-15.614410128791405, P6=-7.494473535183476, P7=-6.473923698544445, P8=-5.393300698605594, P9=-5.623878827876655, P10=-11.06790833569457, P11=-5.644013354624683, P12=-3.7943782702128876, P13=-2.119558111596261, P14=2.8753413650020936, P15=4.378368102478296, P16=5.658653379855187, P17=4.269644400244631, P18=-3.094943484820423, P19=4.826268989723891, P20=-2.321209413942538, P21=1.2847589030879214, P22=1.7420404220642243, P23=2.541022751695539, P24=1.8448662225050334, P25=4.697051203873929, P26=1.1665868455965072, P27=2.0791981691970043, P28=-0.34624278084630866, P29=0.15862394514775593, P30=-0.27377305865891793, P31=2.417742279762837, P32=-2.142818309146257, P33=0.5729556728557177, P34=-1.6965719486451083, P35=0.2597790506040369, P36=-0.38389375028210615, P37=-0.5666848599513242, P38=-0.7745022204798864, P39=0.05778931083291825, P40=0.7116165272658326, P41=0.7083972428078917, P42=0.7045668650227838, P43=0.7218509084766568, P44=0.7481633649369539, P45=0.7441472744540643, P46=0.6883579880132322, P47=0.6653236050391669, P48=0.6331029473852299, P49=0.6317024200991397, P50=0.6860208074642147, P51=0.7041536020463671, P52=4.673985348241815e-06, P53=1.701465604176458e-05, P54=1.1699479427387512e-05, P55=1.3504181812274945e-05, P56=2.0413184663471423e-05, P57=7.421403148515498e-05, P58=0.00023380846171631073, P59=0.0006866782872777094, P60=0.008154144998321398, P61=0.07921196642103764, P62=0.11953864390539179, P63=0.052375721015694064, P64=0.027581010641521596, P65=0.07765299187301423, P66=0.1246578857447626, P67=0.08110363385692029, P68=0.07231252736990754, P69=0.08938722438294333, P70=0.11703365136593011, P71=0.08633144870380335, P72=0.08825922502800683, P73=0.0571385873094499, P74=0.03735001730499495, P75=0.038045359694804125, P76=0.016103915212629463, P77=0.011617231290539085, P78=0.0165417011571583, P79=0.03129116372449789, P80=0.11139764443287947, P81=0.22798836466278344, P82=0.2012751563905983, P83=0.18839619084593903, P84=0.25538409281364005, P85=0.16717033527552644, P86=0.13363251818785443, P87=0.1814016268954428, P88=0.26922994155200686, P89=0.1726096874530603, P90=0.1789688746308886, P91=0.34626067164439345, P92=0.20736677626757538, P93=0.03174342967795331, P94=0.020684844331945222, P95=0.012327932606638474, P96=0.009703365831784242, P97=0.01080409210726768, P98=0.018643815643416446, P99=0.02776840985602508, P100=0.0448731777477864, P101=0.06118238228796608, P102=0.017234478547545118, P103=0.03211494322032649, P104=0.0254462562919604, P105=0.02725973014541097, P106=0.04527711524029209, P107=0.013528392162386752, P108=0.008299468223015999, P109=0.013704736110248293, P110=0.018402173920108338, P111=0.01293901904650209, P112=0.01702790768163948, P113=0.010538293585994759, P114=0.015990631357346845, P115=0.024345962272060092, P116=0.027904164354228163, P117=0.03688259006201318, P118=0.00789258172821266, P119=0.009515331032767404, P120=0.012430500304584839, P121=0.0023700565678632488, P122=0.0027198085697751296, P123=0.0026674217756404725, P124=0.001033541178017039, P125=0.0011887258058609009, P126=0.0009516559212251673, P127=0.0017322463578013656, P128=0.0014511961500317833, P129=0.0016943763567500271, P130=0.003034826886241229, P131=0.0026289609543177755, P132=0.004334969604007907, P133=0.0021179855547146143, P134=0.0024984636779578546, P135=0.002562524582983212, P136=0.0008794193726520811, P137=0.00037749758631444206, P138=0.0005375219701760264, P139=0.00017895716277452591, P140=0.00020214686397207005, P141=0.0002416149180326063, P142=0.0003971125792247707, P143=0.0006395676097991427, P144=0.0007973598045929746, P145=0.0005038251564670497, P146=0.0004864838065625672, P147=0.00022734703347572054, P148=0.0001365181044438998, P149=0.00012602702553601877, P150=0.00016647023820033966, P151=0.00012835562407376356, P152=8.848312423997582e-05, P153=8.183144660896747e-05, P154=0.00010063806984980646, P155=7.14250763911851e-05, P156=0.00010062928736167862, P157=0.00010027564039004644, P158=7.7080272573803e-05, P159=0.0001522729329158945, P160=0.00014557465341751032, P161=8.785384277398986e-05, P162=9.346211255261268e-05, P163=6.521861632099078e-05, P164=5.080287094544277e-05, P165=5.7593110705364264e-05, P166=6.810340751735905e-05, P167=8.481160953492767e-05, P168=8.705617138107756e-05, P169=0.00011503182005975673, P170=0.00012057612277151609, P171=5.64869747018842e-05, P172=5.768703700394034e-05, P173=6.212772931096845e-05, P174=5.8295121178475345e-05, P175=5.346799874465992e-05, P176=4.336745641323592e-05, P177=4.3695707430555326e-05, P178=2.0026918233130832e-05, P179=2.0130925051095392e-06),Row( P0=-552.1799749213599, P1=43.61814648216894, P2=-26.408436625965578, P3=-4.734569888951858, P4=-13.84454169046908, P5=-21.89143824549215, P6=-13.540707907495568, P7=-4.344683260969746, P8=-1.3217647474036647, P9=-6.639194774351902, P10=-13.797781499029947, P11=-7.183448373326447, P12=-5.715807004454872, P13=-10.963826777546666, P14=-5.366352455032748, P15=-3.78702394057011, P16=-4.56392397594369, P17=0.12137162048762624, P18=-2.907323573765897, P19=9.633652119899086, P20=7.23238688642987, P21=8.745477010844867, P22=8.135680903992434, P23=4.009902936679899, P24=2.5190374136264597, P25=3.4887177884412495, P26=-2.3005005335163573, P27=0.3223232369060636, P28=1.0022792117932626, P29=0.6257156497131638, P30=-2.0029840925887488, P31=0.24601321007960034, P32=-3.567201435762227, P33=-1.741741414439631, P34=-3.539116918001013, P35=-1.8225665132233841, P36=-2.335429548071046, P37=-1.1776987990300827, P38=-1.0384156936273332, P39=1.8533840934178587, P40=0.4336035487350193, P41=0.4254353588104944, P42=0.43632669594508705, P43=0.4567446437122037, P44=0.5023755603991681, P45=0.5493514300949923, P46=0.6323307746483442, P47=0.6503099532997431, P48=0.5849753921042548, P49=0.546290013589683, P50=0.4900153977192082, P51=0.4435331059554569, P52=5.54435576134505e-06, P53=2.1228826446128408e-05, P54=3.102255722744888e-05, P55=3.5589221745247396e-05, P56=4.055880357407915e-05, P57=7.872413575746946e-05, P58=0.0004104639896521059, P59=0.0029146083843802108, P60=0.021708980953536888, P61=0.11915670630003258, P62=0.5167425469562227, P63=0.8207531301524039, P64=0.44048420711720454, P65=0.2492504667521566, P66=0.0464583747812912, P67=0.006724450654146613, P68=0.004055083048590079, P69=0.006267072287340738, P70=0.009400788804561126, P71=0.02805061057854805, P72=0.06878802008262448, P73=0.10504549480010232, P74=0.16025260230924923, P75=0.22816811008817378, P76=0.28434593354811, P77=0.47226185855787506, P78=0.5805954812147465, P79=0.34758939937621386, P80=0.09873571856470528, P81=0.03667777799776724, P82=0.03195506583352115, P83=0.06025830424521996, P84=0.1083404683530018, P85=0.12099234088544668, P86=0.13294386137539468, P87=0.1660268002159056, P88=0.30576788699305846, P89=0.2138743198606718, P90=0.4266894281322986, P91=0.43789207969818983, P92=0.17939340724455902, P93=0.09729066856654352, P94=0.04350249403564673, P95=0.013790371290350165, P96=0.009472007328794345, P97=0.032114010000622265, P98=0.0372055051670925, P99=0.09596716853933329, P100=0.043163540025993055, P101=0.004069499091691661, P102=0.00392222067491353, P103=0.015358781250495486, P104=0.030578570785422355, P105=0.030861037825734218, P106=0.027307253484162075, P107=0.01220017480605678, P108=0.0047889258067989274, P109=0.006632195357349468, P110=0.009778347683740637, P111=0.03475829508522453, P112=0.07673633928019982, P113=0.05355860311385344, P114=0.036968784436416866, P115=0.01701742035613793, P116=0.01640052615048468, P117=0.009626211897332877, P118=0.023045516270888702, P119=0.015488924305849963, P120=0.01666098742389443, P121=0.03556934738134325, P122=0.013821807695459418, P123=0.007278467973498573, P124=0.0068195834291524555, P125=0.005485212531282756, P126=0.002764699268469078, P127=0.0033555988234248157, P128=0.0023207749425188467, P129=0.0019198427084682252, P130=0.0030945786720705235, P131=0.004952055827702193, P132=0.004144478186489857, P133=0.004088276556976977, P134=0.0039473553920164476, P135=0.004643613020516037, P136=0.003210818027208488, P137=0.0023144625922463594, P138=0.0011550833834093098, P139=0.0013128728522442488, P140=0.0011595621458218592, P141=0.0015686810056130334, P142=0.0017683048518047657, P143=0.0024477693579100655, P144=0.0028559527479391986, P145=0.001848714936054966, P146=0.0011309565026814993, P147=0.0007211102815293083, P148=0.00040807466109445936, P149=0.0007764118209899615, P150=0.0010836579791164353, P151=0.0011790981584493927, P152=0.0013915494906413988, P153=0.0016533646304873174, P154=0.0020653930319282743, P155=0.0017455872667926126, P156=0.0024913504502756764, P157=0.0026496221755909414, P158=0.0029044016531393803, P159=0.0036185632786933136, P160=0.003287114525007319, P161=0.0033777404311858103, P162=0.0033744905365465433, P163=0.0025736312900623342, P164=0.0014270035421710028, P165=0.0013816621152414926, P166=0.0015143932523740702, P167=0.0013654434020703803, P168=0.0015289999493495877, P169=0.0015350525492982784, P170=0.0015491789202801648, P171=0.0008976038358276787, P172=0.0009076452832997131, P173=0.0007653790492279167, P174=0.000604445649012823, P175=0.0006603105877864557, P176=0.0005985760137445332, P177=0.00042180933897342007, P178=0.00034027063730939606, P179=5.1417644186862144e-05),Row( P0=-627.079831786449, P1=37.96638776554634, P2=-17.216524381963737, P3=-3.122431033982719, P4=-12.509840874777254, P5=-17.97713035695719, P6=-8.35270997377431, P7=-6.005800812840197, P8=-4.469440933167482, P9=-5.626094791062716, P10=-11.716409576983278, P11=-5.508573184582909, P12=-6.016279204225139, P13=-8.332058638594702, P14=-5.541280328843487, P15=-5.784160041953633, P16=-2.4202972931950315, P17=-0.4911009679573972, P18=-1.1128599640805301, P19=9.986902329446366, P20=5.490188781285336, P21=8.665610421844036, P22=5.729921349875002, P23=2.904589798775589, P24=-1.7088214828467174, P25=1.5076843943423568, P26=-0.5680633708555142, P27=0.796740444095299, P28=-0.2898281340132097, P29=-0.27910681578350216, P30=-3.641344673706777, P31=-0.5332706360385977, P32=-4.28666624909656, P33=-1.2872004734345674, P34=-2.5816104779215507, P35=-2.7894418597283366, P36=-1.2543102687956575, P37=1.2283819493254664, P38=0.6431668921810032, P39=1.1275715778088613, P40=0.5550023084700103, P41=0.5872279038049638, P42=0.6081257168933263, P43=0.6439894736369668, P44=0.6818304443377761, P45=0.6770096373808316, P46=0.6977967941950138, P47=0.6732888980190034, P48=0.7147671166676839, P49=0.7198662732030368, P50=0.6189851719215, P51=0.5400650782866139, P52=1.2137566726696684e-06, P53=4.5920940305626656e-06, P54=1.3762273690525948e-05, P55=2.636453171944847e-05, P56=2.6800638136495344e-05, P57=3.544074679197675e-05, P58=8.092534024499509e-05, P59=0.00035785248759790983, P60=0.002765890489736894, P61=0.025257035224696966, P62=0.10047088777081278, P63=0.13852125177580313, P64=0.13789775356420683, P65=0.16234536816522474, P66=0.05987399573956832, P67=0.005094607085965793, P68=0.0008169969361697898, P69=0.0009634196278122362, P70=0.0012197416007149796, P71=0.002235455903104174, P72=0.008714344085733135, P73=0.015222316643526976, P74=0.051177610660025095, P75=0.061460555022791664, P76=0.04444086106413304, P77=0.08923124638864156, P78=0.1364711566049814, P79=0.2237825224919447, P80=0.2720909469154323, P81=0.09890149919693754, P82=0.014335717767797143, P83=0.015188949482384756, P84=0.02767514251908039, P85=0.04318110986934542, P86=0.08398941319724035, P87=0.06554710191096311, P88=0.06818959205916614, P89=0.0758004679464007, P90=0.10190351958535146, P91=0.08682852077675689, P92=0.06624413274633702, P93=0.015436722924016329, P94=0.012029642663052807, P95=0.008082614939538128, P96=0.00388466499466895, P97=0.0099370563574019, P98=0.036177187133780854, P99=0.04770849226706886, P100=0.02384075233585947, P101=0.00640414049245466, P102=0.00412578331480106, P103=0.004637967846536476, P104=0.004912201306636218, P105=0.005897518568169015, P106=0.007409997672934259, P107=0.004618802340174528, P108=0.0015985644634949326, P109=0.0024518982752241934, P110=0.008651978577088288, P111=0.02172445147972204, P112=0.023447923284448037, P113=0.012024140942411485, P114=0.003087733487953825, P115=0.006416903513012386, P116=0.010715853287822437, P117=0.006278711660469737, P118=0.004498514718083917, P119=0.00931179840989138, P120=0.006240290355215617, P121=0.0016065899860761023, P122=0.004296321034988228, P123=0.0023558879906273675, P124=0.0005600981488147402, P125=0.0009372674071250554, P126=0.0011175500747245823, P127=0.0010786782820445618, P128=0.0006325931324656243, P129=0.0018328873881337795, P130=0.0012164396408350525, P131=0.001004259914864497, P132=0.002249285818244166, P133=0.0015066692280726586, P134=0.002208931094168335, P135=0.0026021168586279875, P136=0.0011552714810434703, P137=0.00045719468927764236, P138=0.00031293335162023704, P139=0.00017373633190174848, P140=0.0002805900499239854, P141=0.0002863336733850328, P142=0.0004621160105296954, P143=0.0006866802792448431, P144=0.0005969311732780581, P145=0.00035155546221658546, P146=0.00018079822055543657, P147=5.87401313018675e-05, P148=7.832938763530366e-05, P149=0.00013175869739136064, P150=0.00011247666306004758, P151=0.0001631303444422634, P152=0.00030911186370204634, P153=0.00034936202878370646, P154=0.000333327680569865, P155=0.00047492858738848454, P156=0.0005026309811293396, P157=0.0004762603234135937, P158=0.00043662927394351843, P159=0.0003372107283208796, P160=0.00034458428367544765, P161=0.000324759118663596, P162=0.0002766490699723264, P163=0.00033442096930810875, P164=0.00028746097964447114, P165=0.00023746944880564738, P166=0.00016306682866738893, P167=0.00022718760540411343, P168=0.0002503317451874509, P169=0.00014000961869108021, P170=9.883392882543221e-05, P171=5.166570200480393e-05, P172=6.890510868052727e-05, P173=8.522904812852214e-05, P174=0.00010428615831944205, P175=9.231330249529831e-05, P176=5.861203659761802e-05, P177=4.395085902430421e-05, P178=2.1455769024449817e-05, P179=2.804184767737555e-06),Row( P0=-593.6081157890277, P1=43.98724975810598, P2=-8.78975481134912, P3=-2.758120164745269, P4=-11.557611561228033, P5=-14.445916831671212, P6=-12.899057459916548, P7=-4.968531862552481, P8=-5.494736133653503, P9=-4.185197840047491, P10=-10.038316205117493, P11=-7.7136285226657595, P12=-6.01744215461116, P13=-10.150227676809045, P14=-6.424859754860664, P15=-3.8995094176538716, P16=-3.6816701977673847, P17=-0.9129165860276828, P18=-2.485033186422374, P19=2.1918541149940567, P20=-0.2168695599440372, P21=2.5318320809266677, P22=1.4987982248876919, P23=3.481845699059991, P24=2.9282615218595547, P25=5.170374289677661, P26=0.25817076563440006, P27=0.7124163069201461, P28=0.428940864584828, P29=0.8059092690605572, P30=-0.124169956230924, P31=2.485379322008526, P32=-0.6959756844677398, P33=-0.8009066291873727, P34=-3.139137190455673, P35=-1.1212817647492836, P36=-0.9528415889595647, P37=0.43836312063732474, P38=-0.6178070470863073, P39=0.5123994534223761, P40=0.7391824718544482, P41=0.6922961428197031, P42=0.6699681176084202, P43=0.6762476521029618, P44=0.7531453139850025, P45=0.8022237390158466, P46=0.7812139941629438, P47=0.7280067918023854, P48=0.723314369682041, P49=0.7304095054039453, P50=0.7265620797098021, P51=0.7257603367718846, P52=5.674915756856146e-06, P53=7.529551203982881e-06, P54=1.577774779471741e-05, P55=4.099324351979317e-05, P56=0.00011791956268492295, P57=0.0004453159597625189, P58=0.001093082550660927, P59=0.006125288583199726, P60=0.032324312149586115, P61=0.12361576436869072, P62=0.2859038496450846, P63=0.2495817443652874, P64=0.1769016788188579, P65=0.22488456597894738, P66=0.14271906903931855, P67=0.04827353665708784, P68=0.01797925588721168, P69=0.01500918251720365, P70=0.018087828241516002, P71=0.02351661475355499, P72=0.03720525446070921, P73=0.056801996999640975, P74=0.10309967697037573, P75=0.08424469716504393, P76=0.05965416142916181, P77=0.09873159746230961, P78=0.12299776132327121, P79=0.13736948602866622, P80=0.12876790548151823, P81=0.19698610265904384, P82=0.1759910777007669, P83=0.13339655416342475, P84=0.19099354356263934, P85=0.11875225746826396, P86=0.05651202571589935, P87=0.036924745112371475, P88=0.027794276605812608, P89=0.01863989723176156, P90=0.037072566648013935, P91=0.11878268578058891, P92=0.08714707308362997, P93=0.03531078312902406, P94=0.04156293606757973, P95=0.027101183655773672, P96=0.011403140000503584, P97=0.009772383531928676, P98=0.008252307854303354, P99=0.012933145303880603, P100=0.02224277290945554, P101=0.013613519542658518, P102=0.041557314372466826, P103=0.05670608780123987, P104=0.015344172193687266, P105=0.007114153243493759, P106=0.012546182060081203, P107=0.023498738515953792, P108=0.0121335376544973, P109=0.011303324173738485, P110=0.013018787688557219, P111=0.00555227008484115, P112=0.020142821050434655, P113=0.021832597528005055, P114=0.018008999862351768, P115=0.017094335376210433, P116=0.018067744216048922, P117=0.005172304229979394, P118=0.004827929854025223, P119=0.004655142892582531, P120=0.00317943444743008, P121=0.0052442460624947335, P122=0.005103699278133892, P123=0.0032831561675551063, P124=0.004041923390770366, P125=0.0038027232506826505, P126=0.002135027010689295, P127=0.002597485677508269, P128=0.002155023207940323, P129=0.000719961652600767, P130=0.001167542913862227, P131=0.0020529601688969758, P132=0.0017769748275059003, P133=0.0026053580223462333, P134=0.003818726591747713, P135=0.0031450226375989254, P136=0.002668369112136732, P137=0.001502848069192185, P138=0.0010777615493131525, P139=0.0008949527290188938, P140=0.0007513190968024201, P141=0.0014534325230692994, P142=0.0012925385594289326, P143=0.0009282732980118067, P144=0.0009918538305648333, P145=0.0008008777168684537, P146=0.00047707405038482746, P147=0.0004893158671523882, P148=0.0005349410946935714, P149=0.0006136472963762336, P150=0.0010009632557647248, P151=0.0009179529211045303, P152=0.0007175957215309556, P153=0.0009695520828508693, P154=0.000973593266827318, P155=0.0007531938183012645, P156=0.0008813504698800367, P157=0.0008205495100576788, P158=0.0008862999389240301, P159=0.0008208242044416313, P160=0.0012766964743952665, P161=0.0012494810334529247, P162=0.0012226383458358505, P163=0.0013087114746141875, P164=0.0012099126109877338, P165=0.0011297280261004861, P166=0.0008160669403334981, P167=0.0011303861924974747, P168=0.0013359555964292764, P169=0.0012492624558586914, P170=0.0005651929161860794, P171=0.0005643527566335478, P172=0.0005901270149310392, P173=0.0005159492167361788, P174=0.0004180878857324561, P175=0.00036482979034755753, P176=0.00020025389212850405, P177=0.00015789008861859534, P178=9.695403993906995e-05, P179=1.8988128832189012e-05),Row( P0=-698.6367988590172, P1=65.19575530032067, P2=-6.081942280632911, P3=3.374423452457615, P4=-4.009958309168644, P5=-12.140393815982735, P6=-5.318161194331857, P7=-1.5138702611382546, P8=-2.055758402763744, P9=-3.2413026682659085, P10=-5.083441843925993, P11=-4.169918414557503, P12=-0.004610809664044456, P13=-7.217256857451255, P14=-3.4111166624583737, P15=-2.5226864479090803, P16=-2.4619265894493134, P17=-1.5856827062504224, P18=-4.050411675670755, P19=1.1526649870504668, P20=-3.4861681717324715, P21=-0.436196382293084, P22=-0.05058204026574204, P23=3.00841810580473, P24=2.2082118680487812, P25=3.374905127042994, P26=1.189838433479915, P27=2.592539478472843, P28=3.07963150171989, P29=5.038742575039404, P30=2.680461420031988, P31=4.28051302278032, P32=-0.6433979895096489, P33=3.220406470651448, P34=2.861544207592145, P35=2.4204853841166116, P36=1.1977068356255727, P37=1.4386527170568957, P38=-1.6635669848207786, P39=-0.7790663443721626, P40=0.7018540544245063, P41=0.6935372715601895, P42=0.7013820899930908, P43=0.7188111997380977, P44=0.7091711804315391, P45=0.6914117191363797, P46=0.7152199949103998, P47=0.6734769865090623, P48=0.6451615890106428, P49=0.6886071664151057, P50=0.6913308643660143, P51=0.6690344442792324, P52=5.444308740480368e-06, P53=2.0411997920424553e-05, P54=1.2406470825690131e-05, P55=8.138625762503276e-06, P56=1.4112695530255574e-05, P57=0.00047170369720244337, P58=0.007252564204265913, P59=0.01798403382863605, P60=0.029714819970197943, P61=0.029335469670351645, P62=0.02690440426038119, P63=0.03141709163541677, P64=0.005669646159975636, P65=0.0008309855152218087, P66=0.0018425567923260739, P67=0.003195414259441179, P68=0.0053790419597583026, P69=0.012244306239574987, P70=0.027477840027475248, P71=0.015233004100747733, P72=0.0021903292234577193, P73=0.0022056700267718402, P74=0.008359697969669136, P75=0.016297280207224662, P76=0.008352207247565104, P77=0.007152413136301827, P78=0.009345025684612065, P79=0.026724625511796208, P80=0.045898855365720796, P81=0.01933512532593705, P82=0.005559073079963324, P83=0.0030529775498081284, P84=0.005391353971423461, P85=0.0077311189372849904, P86=0.011622756037933927, P87=0.009788644647164257, P88=0.002454313038555915, P89=0.0013545861293164114, P90=0.0018782417310872176, P91=0.0030074223669420445, P92=0.0025893097077987395, P93=0.0019011283680090542, P94=0.01076774992190711, P95=0.010969187609496715, P96=0.0028887299039883395, P97=0.0006789866929004124, P98=0.0006346190648327665, P99=0.0011569251223687402, P100=0.0014496051996547587, P101=0.001459836026166211, P102=0.0010639391154947963, P103=0.0004544057276964538, P104=0.0007756941009756389, P105=0.0007341289434558136, P106=0.0008140328086406672, P107=0.0010428429504906254, P108=0.00039386441459619956, P109=0.0002868074205627433, P110=0.0010856049549478426, P111=0.0037120316214684255, P112=0.00315513870507628, P113=0.0007329391407477772, P114=0.0009999599795054177, P115=0.000976719762943047, P116=0.0017013104210141866, P117=0.0007449557784223144, P118=0.00011548607178302877, P119=0.00037460885252763406, P120=0.0012289003771145768, P121=0.0006720864150506027, P122=0.00018554756899538427, P123=0.0001583406218180958, P124=5.5421562855087296e-05, P125=0.00013643928455240497, P126=0.000126499695698936, P127=0.0002512435242721181, P128=0.00017778944386687585, P129=0.00026657076730551675, P130=8.215785923950353e-05, P131=0.00010824978645582349, P132=0.00020947471206631148, P133=0.0002861522674129819, P134=0.00018197303976569715, P135=0.0001719954362802656, P136=0.00010521866759688054, P137=4.692990594826634e-05, P138=7.771567709754268e-05, P139=3.4731768316059066e-05, P140=3.4395663179254884e-05, P141=3.2764780006106925e-05, P142=4.13510283357293e-05, P143=3.93413014112674e-05, P144=2.4674577041629914e-05, P145=1.70705880405613e-05, P146=1.0252909358522823e-05, P147=1.387426991461691e-05, P148=1.1065400650550385e-05, P149=8.138694961588211e-06, P150=2.350200509818457e-05, P151=2.5832785285337272e-05, P152=1.6698382235562136e-05, P153=1.6940602499367197e-05, P154=3.593436546914102e-05, P155=2.6952612080392808e-05, P156=1.7332203195656315e-05, P157=3.732940745702417e-05, P158=3.5101681695551e-05, P159=1.996938598647106e-05, P160=1.534192659864059e-05, P161=1.54749486040855e-05, P162=1.4873055135236469e-05, P163=7.979074897781686e-06, P164=4.93131369898606e-06, P165=7.311205436557898e-06, P166=8.24164276565239e-06, P167=1.6229476224389438e-05, P168=1.9074228785427178e-05, P169=1.7192884920946854e-05, P170=1.120635537231963e-05, P171=7.565527920651487e-06, P172=4.316166485301919e-06, P173=4.8123504884011834e-06, P174=4.142618151811705e-06, P175=4.497352895749843e-06, P176=4.737593878069806e-06, P177=5.177962808670064e-06, P178=2.5507564705390247e-06, P179=2.0806761785244326e-07),Row( P0=-701.1860429689751, P1=50.91224749290679, P2=-10.554549527201116, P3=4.204993741371593, P4=-5.3394620817125436, P5=-11.167143930513927, P6=-5.832102216930721, P7=-3.7702714764219616, P8=-0.9282492200394833, P9=-1.4868588307217219, P10=-8.1498536531245, P11=-4.474642377511774, P12=-1.3939328732492742, P13=-7.4793958915672745, P14=-2.519883213699162, P15=-1.9234510538491008, P16=-3.45686378090542, P17=-1.5735093662140283, P18=-5.052808317418415, P19=-0.5131248051818168, P20=-4.556254923994238, P21=-0.739730579036078, P22=0.8187953460491796, P23=3.6229346498280646, P24=3.8527623719625814, P25=8.026530918685287, P26=7.235641567886202, P27=5.234310525598086, P28=5.077470230105576, P29=4.221829518774788, P30=0.8789808662637607, P31=2.2175991130326684, P32=-0.5369939127681584, P33=1.6974021228130598, P34=0.8525850471848354, P35=0.7297421460277065, P36=-1.8400521632572275, P37=-0.3972039079069062, P38=-2.441287692219612, P39=0.910418842320804, P40=0.661074274143229, P41=0.6634475451980281, P42=0.68764580946068, P43=0.7807267905871887, P44=0.8367879007781795, P45=0.7891817055114977, P46=0.7241721869421346, P47=0.6818619670249252, P48=0.6489444392419798, P49=0.6246358149712384, P50=0.6434426873949051, P51=0.656296665834626, P52=6.041159698088485e-06, P53=1.9683664649446613e-05, P54=8.495690661527888e-06, P55=7.31906944602685e-06, P56=1.5744235429718696e-05, P57=9.7246440835839e-05, P58=0.0015130673491671222, P59=0.009802669068708345, P60=0.036190583392796405, P61=0.07324827899750684, P62=0.05924313894078596, P63=0.03369084435106249, P64=0.0068607744447572215, P65=0.001233285557323692, P66=0.0017324862800841335, P67=0.0037011514752645722, P68=0.0063382348994715934, P69=0.012094339719152808, P70=0.02060852144125523, P71=0.031663895616474085, P72=0.017375690091606874, P73=0.010946653410168948, P74=0.017155946597151434, P75=0.015217201574660428, P76=0.006579463404306322, P77=0.0035007140866939027, P78=0.003347584856921928, P79=0.0037009297014602214, P80=0.008821899003011936, P81=0.026040070985109173, P82=0.020514878708359444, P83=0.007631123002400704, P84=0.012404621336069708, P85=0.00915842194514967, P86=0.0035943425449794595, P87=0.0033047675165429335, P88=0.005053524024566277, P89=0.007322839413979833, P90=0.010631978006921118, P91=0.008556641189240911, P92=0.0027657858983863464, P93=0.0026939950788233136, P94=0.011766423914863423, P95=0.016924900800076876, P96=0.011680945449039864, P97=0.0053751864506979725, P98=0.0013556946087579713, P99=0.0010135123782165443, P100=0.0013842033529428872, P101=0.0008052970402809209, P102=0.0026750701234888753, P103=0.004893909751371318, P104=0.0009273144740402668, P105=0.0014831449184092053, P106=0.001996973411124349, P107=0.0011253162143617993, P108=0.0006114894403867493, P109=0.0022172263542122247, P110=0.005436276599446329, P111=0.0027121985818514844, P112=0.0021958970225592817, P113=0.00628771554049021, P114=0.005297009243376962, P115=0.002237995743628574, P116=0.0017373069295439191, P117=0.0008418103478466274, P118=0.0013371235625357172, P119=0.0007846846830912897, P120=0.0009661147759616537, P121=0.00112594455287213, P122=0.0012160419537552683, P123=0.00032657017212651756, P124=0.00011930775057237106, P125=0.0002911481110772162, P126=0.00020976297089201945, P127=7.305105803567275e-05, P128=0.00013512271211706787, P129=0.0002116541311139038, P130=9.584377814835639e-05, P131=0.00024973467028132105, P132=0.0003006383795523251, P133=0.00042090129611314995, P134=0.0005755598690157661, P135=0.0004328154271840058, P136=0.0003305366033697729, P137=0.0002179986479196342, P138=0.00033185862381088856, P139=0.00033664723377470487, P140=0.0002241020894758222, P141=0.00016540783133511044, P142=0.000155662293115126, P143=0.00010267524049197424, P144=0.00010598170016873607, P145=7.605344745392934e-05, P146=4.165271209994103e-05, P147=1.756566344836783e-05, P148=3.2645363633754825e-05, P149=5.031400974874089e-05, P150=7.713617418179991e-05, P151=5.595314616984922e-05, P152=5.369370589969415e-05, P153=6.052461404030606e-05, P154=4.845540719588482e-05, P155=6.832111102197361e-05, P156=6.267653655895956e-05, P157=6.257737930602854e-05, P158=7.849283294527136e-05, P159=7.823436808964089e-05, P160=6.875062192398137e-05, P161=7.039377613548855e-05, P162=3.934628541719629e-05, P163=3.1483914194949935e-05, P164=2.573800756973382e-05, P165=1.6329873346253578e-05, P166=1.1347113426074043e-05, P167=1.0232092169079216e-05, P168=9.996907164750607e-06, P169=1.806349838035993e-05, P170=1.5088320867651608e-05, P171=2.1174429065916762e-05, P172=2.1126758290829515e-05, P173=2.0555997793438082e-05, P174=1.5852004714753788e-05, P175=1.2058531600365917e-05, P176=1.0325341565414231e-05, P177=1.6067899910723675e-05, P178=8.771173173164183e-06, P179=1.7554195879655508e-06),Row( P0=-607.8379242984494, P1=58.15181755769517, P2=-1.415156314947637, P3=1.7401209745201496, P4=-8.728600839791344, P5=-0.6510795960077693, P6=-5.931136894118943, P7=5.244433130210833, P8=2.9945927587614007, P9=-0.006653660079202027, P10=-5.236336933101608, P11=1.0544864376901886, P12=5.857814443310327, P13=-0.6611754763910294, P14=8.681462708672726, P15=9.165837103335704, P16=4.551916191961594, P17=2.947933716133719, P18=2.9644879519244194, P19=4.09727481745207, P20=0.4890212977975607, P21=3.2999807796875986, P22=2.9050443433458164, P23=0.9421990431645628, P24=0.6204074625241494, P25=4.817650936850049, P26=2.3189547718565002, P27=4.857999837844382, P28=0.8868061914986436, P29=3.7627728318122755, P30=2.3076222771173853, P31=4.197205582146622, P32=-1.1408094227118726, P33=2.8277904177755655, P34=-0.8093728328455205, P35=2.9747856050363235, P36=-0.3189113223933483, P37=1.7193086615514408, P38=1.958587656332146, P39=0.6322826394492707, P40=0.6098103265261364, P41=0.6051799915728038, P42=0.6492462819618601, P43=0.6707749838255416, P44=0.6425934977357257, P45=0.6154597568183077, P46=0.6513674526793843, P47=0.6507971847658636, P48=0.6478623575522809, P49=0.6707363391794199, P50=0.6368149815172292, P51=0.6301933862480245, P52=0.0031589309387200343, P53=0.00044551686560637024, P54=0.00013648534021484007, P55=9.21630729207653e-05, P56=0.0002013622902366241, P57=0.0010394925230360207, P58=0.002766803030021829, P59=0.008282806610223965, P60=0.02404604988498358, P61=0.03735845244691433, P62=0.04429028681272156, P63=0.03435034556776359, P64=0.01164352319859439, P65=0.013732463818231044, P66=0.0330649112976366, P67=0.05538552819154642, P68=0.03451528214557232, P69=0.04644274424554989, P70=0.09855629776184335, P71=0.07277544203119773, P72=0.0417367825268091, P73=0.027546779277682065, P74=0.017887179512001264, P75=0.017905480380761396, P76=0.016142889487930658, P77=0.03586338632432886, P78=0.07277387012845733, P79=0.11575679484679587, P80=0.10295958234284514, P81=0.05579743066394986, P82=0.03959758427826935, P83=0.059221342349876575, P84=0.052334308769055655, P85=0.019049583132425555, P86=0.008418344151491529, P87=0.012555270717369033, P88=0.034075392924122344, P89=0.050124457465997085, P90=0.08234556962482104, P91=0.08561734978853436, P92=0.018033850769413003, P93=0.00496229168961941, P94=0.012197560219877803, P95=0.01619624932548342, P96=0.0037031525165508988, P97=0.001125136737178565, P98=0.0005237125614454465, P99=0.0012787733041079823, P100=0.004513037432253041, P101=0.00845046573946856, P102=0.02630548264965043, P103=0.040493735394640816, P104=0.015372599851181234, P105=0.0014030389233521524, P106=0.001059731515547499, P107=0.001271562034701908, P108=0.0006216243591882193, P109=0.0007790059872147373, P110=0.002406848228892053, P111=0.019388830980525143, P112=0.03613277660904368, P113=0.018868153120913795, P114=0.00034518357138172513, P115=0.00026421832913879633, P116=0.0011991481089749254, P117=0.001490047256922482, P118=0.0015913707788441055, P119=0.0014893247673364923, P120=0.0005643430136128726, P121=0.0028502488194063875, P122=0.007309155515395947, P123=0.001212842685954709, P124=0.0011076981492289843, P125=0.0005589048547464438, P126=0.0002375270772720648, P127=0.00012150685516479758, P128=0.0002869943781116258, P129=0.00039722778562079856, P130=0.00019075277814544048, P131=0.00027661388078595355, P132=0.0005443206269774384, P133=0.000437694183991182, P134=0.00028512905475493145, P135=0.0005304342810085396, P136=0.0005503643670144524, P137=0.00043601316213789096, P138=0.0007210281937971703, P139=0.000555334905586214, P140=0.0003971707981343661, P141=0.00022800295158540812, P142=0.00011342949430985053, P143=0.000216589762485691, P144=0.00015786725340333408, P145=0.00025038429226190915, P146=0.00023267534970404437, P147=0.00013676089418090354, P148=0.00012882581975946138, P149=0.00010857347126976466, P150=0.00014321871966406477, P151=0.0001753065095749281, P152=0.0001612590130292315, P153=0.00020985404005619907, P154=0.0002529842683532967, P155=0.0002750417650496267, P156=0.00023811737356731948, P157=0.0002991113198859627, P158=0.000319801278502752, P159=0.000323298559981952, P160=0.0004060049576327568, P161=0.0004243608570216718, P162=0.00045653604216031176, P163=0.0002645917038257153, P164=0.0001145886812498308, P165=7.40580710745554e-05, P166=0.0001373353620500429, P167=0.00017917237316796917, P168=7.870751687840259e-05, P169=4.081819179626713e-05, P170=4.411976139406512e-05, P171=3.691048715445679e-05, P172=3.0987838963392735e-05, P173=3.866167858253098e-05, P174=4.759992855756359e-05, P175=6.620351349257294e-05, P176=0.00015030674854466934, P177=0.00011165299983905193, P178=2.080974267547396e-05, P179=1.759305688175162e-06),Row( P0=-686.8637425257821, P1=73.73307539752648, P2=5.818914566996706, P3=8.093193310643146, P4=-7.066940838618029, P5=-7.392375960641979, P6=-10.637348441098426, P7=4.480956518336964, P8=3.952806275957292, P9=5.700354626782897, P10=-0.8855431156616617, P11=0.6825888382967993, P12=-0.9550731787299906, P13=-7.506776626487113, P14=3.145655116468467, P15=-0.3698525798803235, P16=2.8981310530873094, P17=-1.3666399465183394, P18=-4.676327470718011, P19=5.447072378907147, P20=-1.3766357889170395, P21=-2.886420464117063, P22=9.208489541116275, P23=6.529724824375119, P24=5.653374274835378, P25=8.501052993178996, P26=-0.18602619706241466, P27=3.7619652460682325, P28=-2.5326179338559425, P29=-2.6228646909879942, P30=0.3757044893762262, P31=7.642576822249237, P32=8.334750185024683, P33=9.003686383014115, P34=2.792025394127749, P35=-1.2196119409085568, P36=-3.8548600346917303, P37=-1.4161315888538146, P38=-1.0706999881441706, P39=-0.6843013380571068, P40=0.7152843654236413, P41=0.5761574429674536, P42=0.48206442813947475, P43=0.4756071965723489, P44=0.5597275668468212, P45=0.6047176765697175, P46=0.583174362965942, P47=0.5833539709225576, P48=0.5038106236253493, P49=0.500599638336593, P50=0.5483489670656331, P51=0.6024887466745528, P52=0.0001083789970136183, P53=7.749725323752933e-05, P54=4.009027162862374e-05, P55=2.070055478502468e-05, P56=1.0767141645486912e-05, P57=3.862599656269607e-05, P58=0.0012337309752680475, P59=0.0056899222609852656, P60=0.003986442406847969, P61=0.017839022401579356, P62=0.035152783380742215, P63=0.023680121969061642, P64=0.004314475490405786, P65=0.006918850777388936, P66=0.010449925025287198, P67=0.01120103189304367, P68=0.0064463062287895044, P69=0.0004400785428420015, P70=0.0007859251431258757, P71=0.003499397987092221, P72=0.014823462496662412, P73=0.01643103636742195, P74=0.01481829458472799, P75=0.020603838848467927, P76=0.01674768018884844, P77=0.0047968967290092784, P78=0.004001922418990476, P79=0.008124688005771487, P80=0.016273850510857172, P81=0.0070556787022326905, P82=0.009850899394972875, P83=0.02863389536110578, P84=0.029106141597823, P85=0.006320822479226747, P86=0.0013110143822772614, P87=0.00021344818827377464, P88=6.11564532013918e-05, P89=0.00011091531118940698, P90=0.0006746041847698913, P91=0.004009341731367014, P92=0.00572222010385343, P93=0.0018334985866688403, P94=0.00011362199567785619, P95=9.044251463054052e-05, P96=8.113210762297514e-05, P97=0.00011338832189160643, P98=0.0007963926577856378, P99=0.0008860283940261307, P100=0.0004771856825319543, P101=0.0004036981886050339, P102=5.1364753662574485e-05, P103=4.266149307793419e-05, P104=0.00043559247271031187, P105=0.0007408771694081894, P106=0.00011778826587834899, P107=6.457563817232472e-05, P108=0.00021373321335033105, P109=0.0006488744885739437, P110=0.0004004107152765492, P111=2.4089681472488014e-05, P112=0.00021464573153581116, P113=0.00021882264337937712, P114=7.237711627367771e-05, P115=3.348597144290967e-05, P116=0.00020366787660047332, P117=8.32581719816494e-05, P118=5.186909596829884e-05, P119=3.766824284643932e-05, P120=8.034965431003906e-05, P121=3.674146135066933e-05, P122=1.5012286368518897e-05, P123=1.77359887964341e-05, P124=1.0192446573642022e-05, P125=9.82649456860514e-06, P126=1.3040443728114306e-05, P127=1.1301525288918583e-05, P128=6.7229030973922e-06, P129=3.2571009510395207e-06, P130=7.589714179063457e-06, P131=9.746498573530552e-06, P132=1.0877043141204408e-05, P133=1.2662309743199295e-05, P134=1.4887728420587776e-05, P135=1.1043549866363345e-05, P136=4.953395139527667e-06, P137=2.9625610851929774e-06, P138=2.4503761580444146e-06, P139=1.7088987050074254e-06, P140=4.2676664517637825e-06, P141=4.324269668976022e-06, P142=8.571382516341647e-06, P143=7.246709340744964e-06, P144=1.1702039417446951e-05, P145=1.2853418804762863e-05, P146=1.0053023795065788e-05, P147=5.472471358611566e-06, P148=6.237852892721168e-06, P149=6.506912150451233e-06, P150=4.487247128051746e-06, P151=2.8857984091241797e-06, P152=3.276259006077167e-06, P153=4.8175614682938025e-06, P154=1.1732316833925107e-05, P155=1.7721642250254242e-05, P156=2.3502612906292843e-05, P157=1.7953725807416287e-05, P158=1.3940933123566334e-05, P159=2.2013491228238758e-05, P160=1.8513118373897575e-05, P161=1.0748523998918698e-05, P162=1.0323705026797713e-05, P163=1.0655325334411272e-05, P164=1.1338101095485436e-05, P165=1.8401096173455795e-05, P166=1.0737532056408382e-05, P167=7.640230768242788e-06, P168=3.5823976730168637e-06, P169=3.613440832580034e-06, P170=2.417379768316562e-06, P171=2.573307777786923e-06, P172=2.7047086633395013e-06, P173=2.1896896726291246e-06, P174=3.101452193040735e-06, P175=3.516582560014011e-06, P176=2.7206869889859805e-06, P177=2.427296686815458e-06, P178=1.204264682296542e-06, P179=1.0414383169809882e-07),Row( P0=-694.539562407963, P1=73.33260819505188, P2=15.428782348211472, P3=16.930653888022636, P4=1.5467409311681022, P5=-0.5856440081234581, P6=-2.1963882944533846, P7=5.715563086300996, P8=4.4299260302514005, P9=5.36817671301706, P10=0.737820577815118, P11=3.508853219365128, P12=-1.2433030425293843, P13=-2.1813614802800383, P14=2.816238997436771, P15=0.1961137861092123, P16=3.4390519568136155, P17=0.2245459774844934, P18=-1.802515429929068, P19=2.412118925663396, P20=-4.883054383729165, P21=-3.4853922854359305, P22=-1.1902045841080577, P23=-3.2328291273325362, P24=-3.7310039044217587, P25=1.1665684287226146, P26=-2.0830043088139947, P27=8.084713543664519, P28=5.241644588235636, P29=5.531204485486974, P30=4.850523316525577, P31=6.287927297491225, P32=4.611330130858757, P33=6.998328718333461, P34=3.212702557961219, P35=5.33040769455593, P36=3.045953679200374, P37=3.6082742808966928, P38=0.30827432402245164, P39=-1.6530919948571274, P40=0.6981049777370635, P41=0.6969726036433006, P42=0.7078147453114632, P43=0.6473180328430931, P44=0.5805553039611797, P45=0.5864117278750213, P46=0.5888822009612905, P47=0.5686892162967274, P48=0.5941103351540887, P49=0.6766774580831573, P50=0.7175717273033074, P51=0.7094224917199369, P52=0.00018532310110801594, P53=0.00021372092379401815, P54=9.95305084165798e-05, P55=6.41704141035511e-05, P56=3.135009764620888e-05, P57=0.00019236297098480443, P58=0.002988881417317326, P59=0.008149425553857045, P60=0.012873737758600448, P61=0.008178416142229085, P62=0.0021530344372348593, P63=0.0002566611047155436, P64=7.182273370918716e-05, P65=0.0011529076814817193, P66=0.006353674719109249, P67=0.008708206061191204, P68=0.010862811110344425, P69=0.02289261966183695, P70=0.0259055438965962, P71=0.011793561443784809, P72=0.001422146365538955, P73=0.001132637643495855, P74=0.0026246199115641322, P75=0.001998908976793027, P76=0.0021940493691786638, P77=0.003422936836866144, P78=0.011816607554668039, P79=0.02081078826314161, P80=0.015626150108744605, P81=0.016160924017128517, P82=0.00732180212214391, P83=0.0014261250169203944, P84=0.003377751032793883, P85=0.0028915075449033475, P86=0.0007032520321404253, P87=0.0012162063165201272, P88=0.0012310221557605638, P89=0.0008009343452596657, P90=0.001133307484920683, P91=0.0007525699752775189, P92=0.00012805595818566687, P93=0.00035080067716552967, P94=0.0031549199938593242, P95=0.002619672405984991, P96=0.0004668856444019803, P97=0.00019972708488179392, P98=8.377837488545883e-05, P99=0.0010508128219034777, P100=0.002227617915253374, P101=0.0004642862704311413, P102=8.635475536168715e-05, P103=3.9372768091636445e-05, P104=0.0006641899835178256, P105=0.001130838599608872, P106=0.0005465754825957949, P107=0.0003056457455696278, P108=0.002404544978059435, P109=0.0031910767455059236, P110=0.0003378850391064843, P111=0.000166488120664099, P112=0.00021299896823373003, P113=5.2782709848854e-05, P114=2.2999855230423048e-05, P115=7.555639761732812e-05, P116=4.006485215290586e-05, P117=2.466028176547199e-05, P118=9.37256142905705e-05, P119=2.797317449401062e-05, P120=3.784596450025459e-05, P121=7.279413551114454e-05, P122=1.3592865196839697e-05, P123=4.654443991539443e-05, P124=3.775335233954721e-05, P125=1.2401104432967882e-05, P126=1.1467982017061583e-05, P127=8.778125478317998e-06, P128=1.3018739591476688e-05, P129=8.476962319465518e-06, P130=5.766713312699885e-06, P131=1.4652644004881435e-05, P132=2.287918800791025e-05, P133=1.5365355903830908e-05, P134=1.899340294243792e-05, P135=1.9821070835590488e-05, P136=1.589113489125403e-05, P137=8.038377201524728e-06, P138=7.130918376208094e-06, P139=6.260356948809661e-06, P140=1.2775137058598617e-05, P141=1.2154686311740241e-05, P142=1.523662794307063e-05, P143=1.4392678780554183e-05, P144=2.108278362811243e-05, P145=2.7734995003532667e-05, P146=2.275959583217642e-05, P147=1.6690417061420316e-05, P148=9.645518922497911e-06, P149=8.988503311804409e-06, P150=1.3127688660451833e-05, P151=2.4243554149489992e-05, P152=1.7937807680192473e-05, P153=1.0080019487783476e-05, P154=1.3322245848659451e-05, P155=1.7716690910785274e-05, P156=2.2622759351881916e-05, P157=2.8469683072534013e-05, P158=2.4161837533210128e-05, P159=1.2166594244590503e-05, P160=1.1915407777940209e-05, P161=7.144167312118152e-06, P162=4.570575864870944e-06, P163=7.199101075669135e-06, P164=9.784473302901708e-06, P165=1.4817197348914355e-05, P166=1.2211762147235956e-05, P167=9.309793259033405e-06, P168=3.580719055719568e-06, P169=3.4225604492569204e-06, P170=3.390564761083422e-06, P171=3.786567627868119e-06, P172=2.9192878790210173e-06, P173=3.3197744781886994e-06, P174=3.467649609659712e-06, P175=3.907288616261865e-06, P176=4.663031248349064e-06, P177=3.4067171465712586e-06, P178=1.3746085197887042e-06, P179=1.713793734415561e-07),Row( P0=-771.8133737147202, P1=81.32359280689845, P2=23.0112802892118, P3=15.50772771148837, P4=1.3832488325171857, P5=2.605117067162123, P6=-4.3244343576823265, P7=8.135802231849746, P8=2.836429301921578, P9=2.256221049736308, P10=0.03132510110078496, P11=2.0444907163314787, P12=0.6250269858376443, P13=-3.1912382271659356, P14=-0.536715951889808, P15=1.7738739593205977, P16=3.326959672142612, P17=2.494351103529705, P18=-1.6584021901335946, P19=3.3032447666974987, P20=-1.468041419704758, P21=-1.3884320209916319, P22=2.784453755269027, P23=2.138171220021775, P24=-0.30799262155460705, P25=2.6148698032059263, P26=-2.1420029080886014, P27=4.500159840345996, P28=1.5164164319950686, P29=1.4823654768678267, P30=-0.28523050326354144, P31=0.9645388265673281, P32=-1.1087349772819728, P33=3.648837933759103, P34=0.3694985038318568, P35=2.8262085954732123, P36=-0.624807490360039, P37=1.1880560864837788, P38=0.7145522006533191, P39=3.2585858750719607, P40=0.6502597522873692, P41=0.6020315786685112, P42=0.6245209459364992, P43=0.6487564964119897, P44=0.6419943416971415, P45=0.6255623472542978, P46=0.6706665543245243, P47=0.7011110827095358, P48=0.7223507859095712, P49=0.6934337402701537, P50=0.642666677570967, P51=0.656535377427979, P52=0.00010829975006647097, P53=0.00015163348580436814, P54=4.732361767678762e-05, P55=4.338668342416392e-06, P56=0.0003891785042168434, P57=0.004497817375712286, P58=0.0038156112034838717, P59=0.002283240007237372, P60=0.002746067082443297, P61=0.003095936978495258, P62=0.002394441376613454, P63=0.00199741107576036, P64=0.00266110983828796, P65=0.0019828847786940093, P66=0.0036086252031260176, P67=0.005197775207021792, P68=0.0027390348903623104, P69=0.003244648746944081, P70=0.0033223500934001257, P71=0.0021591368624109294, P72=0.0010398757210098994, P73=0.001069249023718923, P74=0.0012051643309589136, P75=0.0008383073867636664, P76=0.0012025482632997753, P77=0.0033574876589599514, P78=0.003109418090485516, P79=0.0017610858655343002, P80=0.0011061416801784539, P81=0.0015683456893937078, P82=0.0020376334936946807, P83=0.001735402041309221, P84=0.0012427415868818247, P85=0.0005005383757995289, P86=0.0001408616535755867, P87=0.00012181886672890213, P88=0.00027315484779516974, P89=0.00035721961024794444, P90=0.0009641466004551483, P91=0.0005949847092397234, P92=0.0001462044054862024, P93=6.0488342195141825e-05, P94=0.0001851706541377631, P95=0.0003043764320255777, P96=6.99109202228105e-05, P97=0.0002526645372172747, P98=0.00016557273399344627, P99=6.321294727010603e-05, P100=2.8861610214165605e-05, P101=2.7829519286144318e-05, P102=6.72752369027044e-05, P103=4.285779675901631e-05, P104=1.0557243485455626e-05, P105=6.903610078271324e-06, P106=1.2161790003523007e-05, P107=3.619926064397366e-05, P108=1.8761601544940072e-05, P109=6.017795268850317e-06, P110=5.609052862281899e-06, P111=4.319321385021295e-05, P112=3.233006805612016e-05, P113=1.0322170129860516e-05, P114=1.1907993542705738e-05, P115=1.2739400953725755e-05, P116=1.1935246496278497e-05, P117=6.48557756036536e-06, P118=1.4505987675040902e-05, P119=1.0512927022206095e-05, P120=1.05269579824689e-05, P121=1.0426710557947306e-05, P122=1.0201452068262286e-05, P123=4.256417585889703e-06, P124=5.354930751106552e-06, P125=3.9820143695087814e-06, P126=3.138657298750932e-06, P127=2.395932998630357e-06, P128=2.338175197975437e-06, P129=1.8670977449457518e-06, P130=1.4443529921074688e-06, P131=3.2475656872312944e-06, P132=2.726275654389598e-06, P133=3.0408609296404975e-06, P134=3.6274375448320223e-06, P135=3.097917663754708e-06, P136=2.025438626354215e-06, P137=1.9293811185927503e-06, P138=1.495439160627706e-06, P139=6.042219431015141e-07, P140=9.5539258340576e-07, P141=1.105400357433976e-06, P142=2.2418160770016043e-06, P143=2.6591529653727193e-06, P144=3.692289894676066e-06, P145=4.284414399124777e-06, P146=3.7720594025444285e-06, P147=2.4624027378040795e-06, P148=1.7646218417404384e-06, P149=1.6759617597182956e-06, P150=1.5870121670522935e-06, P151=5.847275897358994e-06, P152=1.0638507693376308e-05, P153=1.2130416098597509e-05, P154=2.100178615920736e-05, P155=2.259845899808667e-05, P156=2.1323718582468054e-05, P157=1.2810868785184933e-05, P158=6.337843632221255e-06, P159=4.171237086221769e-06, P160=3.8878607010475145e-06, P161=3.857855388620613e-06, P162=3.997383907336718e-06, P163=3.688736925932142e-06, P164=3.1059654499358163e-06, P165=1.5806317439150423e-06, P166=2.028786271987334e-06, P167=2.167955788204688e-06, P168=1.39813970277056e-06, P169=1.3332840671869265e-06, P170=1.0896786336154516e-06, P171=5.565376219270696e-07, P172=3.6153015781394887e-07, P173=4.000867532415338e-07, P174=8.115172547749943e-07, P175=1.0763873138791234e-06, P176=1.9439437089765806e-06, P177=1.3320860650273125e-06, P178=3.8693367375236364e-07, P179=6.157947976482442e-08),Row( P0=-712.5168886807692, P1=69.96930448541926, P2=13.791887147220667, P3=14.752750465662492, P4=-1.8478424617531926, P5=-2.566915386721282, P6=-4.209603835242607, P7=6.961572824360453, P8=3.962885994364135, P9=-2.399803548459297, P10=-3.657318912498139, P11=3.739024707071668, P12=-1.5668454001835963, P13=-5.918207853893233, P14=-0.5843279329114339, P15=-2.3008947116190774, P16=2.160310323658041, P17=1.258839118126954, P18=-4.506559295101924, P19=2.308567005284842, P20=-2.24038735902493, P21=-2.5900157328815157, P22=2.6859603946282045, P23=0.20669111683423008, P24=-5.590333946359368, P25=1.4941009317510325, P26=-4.992304244644404, P27=2.1921910931799755, P28=-1.0179188590510853, P29=-0.3266848441832864, P30=-3.348673478789608, P31=0.31944605770740636, P32=-1.7210007246207069, P33=-0.012688112719512557, P34=-0.97138373586611, P35=0.6461579986646105, P36=-0.3343151312936671, P37=3.1188103042040747, P38=2.32679323823207, P39=1.9078190098339731, P40=0.5676646636130803, P41=0.5714425912771861, P42=0.6074670164197248, P43=0.6189182141602232, P44=0.617266537240539, P45=0.6453452897664543, P46=0.7081809066024914, P47=0.7595625460486817, P48=0.7544996008602958, P49=0.7134875588472548, P50=0.6544541809957996, P51=0.5890093122260366, P52=5.411815572908882e-05, P53=4.741155817927808e-05, P54=3.0493732977570692e-05, P55=2.6641265229396403e-05, P56=0.001192685748773013, P57=0.010535254924202334, P58=0.009952673756477503, P59=0.005375258314303021, P60=0.003315375429073361, P61=0.0024501601747254982, P62=0.002708086594679194, P63=0.006038836543255786, P64=0.0069626993195507116, P65=0.008691869000187877, P66=0.014193384938393354, P67=0.008056470902473108, P68=0.0035594743873541296, P69=0.00680540768993581, P70=0.013515625450235578, P71=0.005774355478851431, P72=0.00209578000351137, P73=0.004478095974093721, P74=0.007856421020724067, P75=0.011284694270007284, P76=0.005835532029223751, P77=0.0030002736257424448, P78=0.004076621556007199, P79=0.006630877952425998, P80=0.003993916330685392, P81=0.0020765754892660515, P82=0.0019715204872150323, P83=0.0014977250619187248, P84=0.0008176605226832017, P85=0.0007383990739624105, P86=0.0013964843103320202, P87=0.0023241501945336683, P88=0.0020744898907861034, P89=0.0006349086438593375, P90=0.00024507848189017504, P91=0.00012360201963280488, P92=0.00015836203751588666, P93=5.606651390656247e-05, P94=3.459151327186659e-05, P95=8.193788172392151e-05, P96=0.00019267093238360225, P97=0.00018343410150782597, P98=5.0129379554204744e-05, P99=3.696240067192656e-05, P100=3.3286101587637444e-05, P101=1.5125390003612989e-05, P102=1.3265867464399973e-05, P103=6.514258430733222e-05, P104=5.128325253185953e-05, P105=2.432606017309041e-05, P106=1.1035302538662901e-05, P107=1.4123593749892014e-05, P108=2.9533737650935163e-05, P109=6.787020384769602e-05, P110=2.9249694094620833e-05, P111=5.554652753754132e-05, P112=2.283357407377284e-05, P113=8.191632946882895e-05, P114=4.153232346265498e-05, P115=4.354620811711355e-05, P116=3.817810622303391e-05, P117=3.61391832871523e-05, P118=6.734421062858234e-05, P119=2.9434221040790418e-05, P120=2.4961067191343313e-05, P121=3.714004589493137e-05, P122=3.162226272509996e-05, P123=4.771536439845374e-05, P124=4.898764101602863e-05, P125=1.9881826147440777e-05, P126=1.5290819101375978e-05, P127=1.5917596333643676e-05, P128=1.743127813975441e-05, P129=1.1171123169341436e-05, P130=1.1809975721176115e-05, P131=2.1739664562323408e-05, P132=3.267369230648239e-05, P133=2.791649527512069e-05, P134=2.0208748858119626e-05, P135=1.2521243604929615e-05, P136=5.5664315788695096e-06, P137=3.992801032454017e-06, P138=4.386559842967105e-06, P139=3.1945618895065914e-06, P140=5.164689135443383e-06, P141=6.112021146251119e-06, P142=1.4486899144132946e-05, P143=2.7468941510108963e-05, P144=4.107147828650645e-05, P145=5.9513208864316695e-05, P146=3.345049034502414e-05, P147=2.1469592145857136e-05, P148=3.0036874420232726e-05, P149=3.945071770901352e-05, P150=2.8015787946181174e-05, P151=2.557933346028152e-05, P152=3.629063331189407e-05, P153=7.181851477888503e-05, P154=0.0001736497419707119, P155=0.00013158987492590965, P156=0.0001786698917776712, P157=0.0002449981509980409, P158=0.00021405629221828794, P159=0.0001299802357512028, P160=8.846296234461427e-05, P161=4.6135586617476814e-05, P162=4.397928529511092e-05, P163=7.926382851605285e-05, P164=9.949575606913358e-05, P165=6.646627587552124e-05, P166=7.440878539072306e-05, P167=2.8886616768383196e-05, P168=1.6762091082763253e-05, P169=1.2765396927450264e-05, P170=1.438494341230254e-05, P171=2.0175111296489032e-05, P172=1.8260905346309197e-05, P173=1.3917448365542641e-05, P174=1.6089293525077207e-05, P175=1.339957596682923e-05, P176=1.6006526837241332e-05, P177=1.17053663332812e-05, P178=4.845049781153747e-06, P179=4.520513606162462e-07),Row( P0=-754.0773742690378, P1=76.21292218738387, P2=21.977693139701948, P3=18.09407289795028, P4=-2.942684905792646, P5=0.1344713278711303, P6=-3.6677666207310793, P7=4.448006542360898, P8=6.381171488268259, P9=4.616165379109431, P10=-4.779415062840901, P11=5.230470129881606, P12=-0.5212193788108849, P13=-2.090936372239902, P14=2.179351492165269, P15=0.39705776866865666, P16=2.366300109166311, P17=2.9149987897832585, P18=-0.021600802781695724, P19=3.89987567104701, P20=-1.374129700829278, P21=2.148029709439293, P22=4.960561937114611, P23=2.185140422472979, P24=1.0662425719396547, P25=2.7621783995078757, P26=-1.5900993992968473, P27=3.532343309470317, P28=0.7536475819467071, P29=-1.0082011213750366, P30=-0.6948461935629058, P31=2.368408732282407, P32=1.3784995252547472, P33=6.876544753732134, P34=5.588965203669762, P35=7.498166213930542, P36=4.464115077426137, P37=4.582925930819372, P38=1.4696241716210003, P39=-0.34344666733829243, P40=0.6667192698504611, P41=0.6353624861812022, P42=0.6162940140163792, P43=0.6297116768941654, P44=0.6587879497793305, P45=0.6777140635124673, P46=0.6628056463630857, P47=0.6156898676151619, P48=0.6278370418539166, P49=0.721564728347531, P50=0.7908820764443171, P51=0.7586657228067414, P52=5.608419961361466e-05, P53=0.00010415100921219814, P54=4.239945789881783e-05, P55=7.556905094496302e-06, P56=2.6479332819649842e-05, P57=0.0004822313188073322, P58=0.0034891704264176943, P59=0.005249925520788488, P60=0.0018213735498532274, P61=0.0016597077261027464, P62=0.002607863682446163, P63=0.005072600567749583, P64=0.002258040744160413, P65=0.003992938481213296, P66=0.010231162022428847, P67=0.008488663421474616, P68=0.0027859676574548183, P69=0.0012270922143071883, P70=0.001801527934204246, P71=0.002959583301660062, P72=0.0034550121694255456, P73=0.00466181961555322, P74=0.006486204649984375, P75=0.0107563329435376, P76=0.007238205221511012, P77=0.002254973962452691, P78=0.001568210961134004, P79=0.0029034510836981353, P80=0.005937495628882692, P81=0.0051441300311495675, P82=0.0032629285750228085, P83=0.002113288295148591, P84=0.0017623920421579542, P85=0.0007447159974154574, P86=0.0013482549380873414, P87=0.0026446112315530465, P88=0.002165897156138683, P89=0.0005664843485385285, P90=0.00023336891512955893, P91=0.00013613918970380826, P92=5.035053444178707e-05, P93=0.00038357482368515776, P94=0.0005895295491001049, P95=0.0002670206677378677, P96=0.00037636038343823137, P97=8.766652877457925e-05, P98=0.00014688378540835304, P99=8.612886794540752e-05, P100=1.4117980494493723e-05, P101=1.0695498776556337e-05, P102=3.337700064747849e-05, P103=4.526736794459064e-05, P104=3.4507028279258774e-06, P105=7.505809952847786e-06, P106=5.299007013442515e-05, P107=3.418568926758524e-05, P108=9.697290651884899e-05, P109=7.17891525970131e-05, P110=9.400642175306219e-06, P111=3.038245514499815e-05, P112=6.401975863640848e-05, P113=9.316367018797772e-05, P114=2.395400664304963e-05, P115=6.905368032247777e-05, P116=1.8514940784935634e-05, P117=0.00011568181853353263, P118=7.826499400682412e-05, P119=3.224724166871809e-05, P120=6.797478470658846e-05, P121=4.88114393752909e-05, P122=8.43424748786626e-05, P123=0.00010686778069843359, P124=9.609282601650464e-05, P125=2.050658085302509e-05, P126=1.8774338140048866e-05, P127=1.8715859863448025e-05, P128=1.5580702828389627e-05, P129=7.216967217035893e-06, P130=6.479696684196113e-06, P131=9.348547923114472e-06, P132=6.545919866291181e-06, P133=1.0623148184273632e-05, P134=2.1791429766013072e-05, P135=2.5726299349359595e-05, P136=2.5813356610130028e-05, P137=2.5378440019817197e-05, P138=6.203774304854917e-06, P139=5.138496156392079e-06, P140=9.930424701693938e-06, P141=3.3539438038872175e-05, P142=4.708722815324681e-05, P143=9.356427749361514e-05, P144=0.00012157884183015225, P145=9.899899754853981e-05, P146=8.470642850811342e-05, P147=6.227895911612988e-05, P148=4.488337792961028e-05, P149=4.70329490844587e-05, P150=2.4737706091808954e-05, P151=2.3722817909886955e-05, P152=4.358343952910636e-05, P153=4.309666904390956e-05, P154=3.900976323172677e-05, P155=3.45405199907249e-05, P156=2.684093899058245e-05, P157=6.576829552794109e-05, P158=3.486659245826705e-05, P159=2.0533359841687295e-05, P160=3.1652430560339316e-05, P161=2.4857063223777907e-05, P162=2.5584539146218885e-05, P163=1.4141529420980851e-05, P164=1.1568431610294408e-05, P165=1.8709139771916988e-05, P166=1.4798959170298483e-05, P167=1.691908760781944e-05, P168=2.3202206815932485e-05, P169=1.3702741625611973e-05, P170=7.57352284886599e-06, P171=5.243557352496299e-06, P172=5.2861606252951724e-06, P173=4.457195997042508e-06, P174=4.922598367280442e-06, P175=5.435522626706699e-06, P176=4.816470132886025e-06, P177=4.5222206812181445e-06, P178=2.1011217435802386e-06, P179=1.925541814020299e-07),Row( P0=-645.8710515728955, P1=36.72122611245864, P2=-6.908611985006744, P3=3.8999680987030723, P4=-14.581018101157985, P5=-7.218922365332916, P6=-13.575529390134717, P7=4.543769800537587, P8=1.3385630225571428, P9=-8.639419961869994, P10=-2.9700517724347484, P11=-1.2872934996336236, P12=-3.307724379691314, P13=-1.8485174692135424, P14=7.942125571228987, P15=8.033542377759591, P16=9.73306212346829, P17=12.333876671248824, P18=6.967045662475927, P19=11.294831999519793, P20=3.6261923484381104, P21=4.260469255673727, P22=7.283327753313723, P23=3.5577955891203024, P24=0.3832719087662059, P25=2.964227088842761, P26=2.714150087821694, P27=2.84742668722787, P28=0.48184864118516396, P29=2.0948720319457763, P30=-3.754000605781663, P31=2.446964816476149, P32=0.5614901069364101, P33=-1.251853180821147, P34=-2.4178473631594253, P35=0.6235344320322316, P36=-0.5899608658575245, P37=1.1538729837923984, P38=-0.3127050537904988, P39=-1.1645273926351207, P40=0.6427410984383205, P41=0.6358265015199986, P42=0.6264735069729412, P43=0.6206659500710582, P44=0.6103421266302826, P45=0.6040723187003769, P46=0.5935788620690017, P47=0.635959416655412, P48=0.6380184693353728, P49=0.6452680814001347, P50=0.7162534736259891, P51=0.7082169173110023, P52=0.0003955358603037457, P53=8.573600856821151e-05, P54=5.0189412157437035e-05, P55=1.351283542704593e-05, P56=9.689781977984869e-06, P57=1.0063680471797347e-05, P58=7.951962321731371e-06, P59=1.337282664801786e-05, P60=0.000128619055770641, P61=0.0007513075392241101, P62=0.004165735826190549, P63=0.01027226473369277, P64=0.010313924673021429, P65=0.03170862408996655, P66=0.05578923871141056, P67=0.052160270460753624, P68=0.033689235580867116, P69=0.032674406471991234, P70=0.04384720452521105, P71=0.022084736972111052, P72=0.003238308017236534, P73=0.001121066546739914, P74=0.002338830044073133, P75=0.006345194262892394, P76=0.007840427793825247, P77=0.00646650900110396, P78=0.010787689928467669, P79=0.01689094773091415, P80=0.019956026688377378, P81=0.024346131983310745, P82=0.0323556223586979, P83=0.022860522611391515, P84=0.024012892775768423, P85=0.02292698003136303, P86=0.017834332594804277, P87=0.011245030958638507, P88=0.01311347338276648, P89=0.01640032003883914, P90=0.02255140761862725, P91=0.01154158526018737, P92=0.00495891753964003, P93=0.005366320403644435, P94=0.014058743497407405, P95=0.009777346422900715, P96=0.00353229685469001, P97=0.004849145907102022, P98=0.0064484346748202, P99=0.010179676388531423, P100=0.008305563939703754, P101=0.0072689017009560635, P102=0.005271119317207777, P103=0.0034972223672351662, P104=0.0004977839507920997, P105=0.0007646995092327637, P106=0.0032146068227745144, P107=0.006544004851127517, P108=0.005184611808014139, P109=0.004807313270255638, P110=0.0032977517085548973, P111=0.011092350655790296, P112=0.01342647734551621, P113=0.002645662912493884, P114=0.00027373644852478324, P115=0.00019797867096614315, P116=0.00018397573765229467, P117=0.0002740917794584238, P118=0.000295703978570345, P119=0.00016328902190547573, P120=8.821106409509218e-05, P121=0.00021263181353689154, P122=0.00023103466441436285, P123=9.354271327163814e-05, P124=0.0001588466302239268, P125=0.00016112030316575265, P126=3.96455570548635e-05, P127=4.104420011242396e-05, P128=9.768290083359204e-05, P129=6.36878000945115e-05, P130=6.504042309946878e-05, P131=0.00010203451842877118, P132=0.00010467596977859175, P133=5.762460442802262e-05, P134=4.486571959960868e-05, P135=5.617248299275634e-05, P136=1.762860386363468e-05, P137=7.286112701277986e-06, P138=8.951551183450284e-06, P139=1.2137104591001083e-05, P140=2.339789294355024e-05, P141=2.7566109200126616e-05, P142=4.262709298586394e-05, P143=7.184414142454548e-05, P144=9.615095292545051e-05, P145=0.00016257273685913045, P146=0.00014493579786985003, P147=0.00011870742869213835, P148=0.00013004057309571007, P149=0.00033782771688821656, P150=0.0004569473157355983, P151=0.0007570063016643947, P152=0.000527072710870939, P153=0.0003859062515287285, P154=0.00037595769940707436, P155=0.0004989062299836878, P156=0.0007702949530965534, P157=0.0007027664478951758, P158=0.0006717565540659033, P159=0.000594542122959569, P160=0.0004168658707543694, P161=0.00035237904684881427, P162=0.00025528005507697524, P163=0.00013460300647774587, P164=0.00024602975605867203, P165=0.0002580516417388267, P166=0.00024138340455240123, P167=0.00018507230967953743, P168=0.00011514899283911262, P169=9.811457618936935e-05, P170=7.728212909221539e-05, P171=6.272722298283666e-05, P172=6.981736188819976e-05, P173=4.7184389376697586e-05, P174=3.827942971822436e-05, P175=3.772030518352366e-05, P176=4.635171156490317e-05, P177=4.800600759001168e-05, P178=2.116152114067992e-05, P179=1.9770697308896355e-06),Row( P0=-662.7807995372385, P1=56.8453731923937, P2=-2.9551128145745698, P3=6.401151443088348, P4=-7.301625905663551, P5=-8.9190623782655, P6=-4.205864352219232, P7=8.596833487665622, P8=3.554386146627287, P9=-1.6163718070848971, P10=-0.2159454031703715, P11=0.23099806017362026, P12=-2.1083226160114172, P13=-3.342898231628249, P14=2.547777164841276, P15=5.087192369647343, P16=5.5411011438723135, P17=9.14531955072463, P18=4.262375133702984, P19=10.252281682422376, P20=6.499274076141056, P21=4.490087844526174, P22=5.243114365565686, P23=8.682285061768562, P24=4.38484947337345, P25=6.795619322720009, P26=7.574826827145591, P27=5.805557507862171, P28=1.9225625359244527, P29=2.7100781418981876, P30=-1.877682718283235, P31=4.387106044114337, P32=2.4858316275364887, P33=2.965434790297657, P34=-2.0674938551242024, P35=0.44549296843049757, P36=1.2087620091211622, P37=2.12254003201065, P38=3.372944604558589, P39=-0.24468633605580598, P40=0.6944284715750911, P41=0.6643246280412195, P42=0.6492321340886964, P43=0.6670305423152469, P44=0.675886864138572, P45=0.6643038196554473, P46=0.662383817198905, P47=0.6672800207549878, P48=0.652075554968727, P49=0.7439985942785645, P50=0.7009360249574709, P51=0.6680837020678254, P52=0.009845004517008017, P53=0.0009593646639463366, P54=8.169790781591462e-05, P55=9.362940390312725e-06, P56=9.46328717747011e-06, P57=8.957830576203511e-06, P58=2.1067753001453634e-05, P59=0.0006513983596683002, P60=0.005331992517303752, P61=0.01111909271099919, P62=0.022598429917527994, P63=0.014497367149162733, P64=0.007814746081654793, P65=0.01931777222248339, P66=0.022168256313487632, P67=0.011986743433295146, P68=0.00875242863345149, P69=0.013176489091081486, P70=0.028274325974591983, P71=0.017978918934781492, P72=0.009360619030526133, P73=0.02098381197904449, P74=0.011213331983526599, P75=0.003013269808130228, P76=0.0016022536134039304, P77=0.00364969694581572, P78=0.01350575811193352, P79=0.04144795323434377, P80=0.032863960227067, P81=0.016496008582859385, P82=0.01164360548111968, P83=0.021544843392831695, P84=0.030512777028415417, P85=0.019465094947452306, P86=0.00756252640447034, P87=0.007121309223884753, P88=0.013866740554944093, P89=0.02209307587176919, P90=0.010787414786709497, P91=0.010404276137388133, P92=0.027275380508349444, P93=0.00853665287127503, P94=0.0012089199657957825, P95=0.0004763534673355541, P96=0.0002722960308678613, P97=0.0005514392233471699, P98=0.00107358918966258, P99=0.0022427902384839224, P100=0.008125557400572463, P101=0.008056526531988505, P102=0.002788112527520993, P103=0.00019783290844217798, P104=0.00029105330514900176, P105=0.0008996167442984822, P106=0.0019067604824716175, P107=0.008584511061315135, P108=0.004159424376256395, P109=0.0001629364322593576, P110=0.00014453445814712978, P111=0.00017579649536852697, P112=0.0001266192614707669, P113=0.00019483017823904573, P114=0.0003899705909698835, P115=0.00011490295862045711, P116=9.073892643630736e-05, P117=0.0009523377021775484, P118=0.00039972823727191286, P119=7.18277459408869e-05, P120=0.00019070046847520665, P121=0.0002451591161796035, P122=6.195197634212048e-05, P123=0.00011547750764626168, P124=9.977077980361265e-05, P125=8.00149988413978e-05, P126=8.291479787033524e-05, P127=6.887426474953033e-05, P128=4.36048132394939e-05, P129=3.3312916478098546e-05, P130=6.878853319323593e-05, P131=0.00010747475020086686, P132=0.00010548132844764823, P133=0.00018503699851096992, P134=0.00019897846860458202, P135=0.00017273066030464832, P136=9.039932189612878e-05, P137=6.435533622716174e-05, P138=3.525368978036749e-05, P139=7.630862505980584e-05, P140=8.844222230800198e-05, P141=5.6852955610431325e-05, P142=5.757073801169921e-05, P143=5.2956009467048475e-05, P144=9.091034986784689e-05, P145=4.635488107343399e-05, P146=7.391917378593891e-05, P147=3.824982160060407e-05, P148=5.2005312199250075e-05, P149=0.00011999412549984138, P150=0.00012954757526206732, P151=0.00012034988038088696, P152=0.00010567475916081715, P153=6.190533168209651e-05, P154=4.6263604019822177e-05, P155=4.24979971808184e-05, P156=9.474526450221575e-05, P157=0.00030657420169592043, P158=0.0003293883504287259, P159=0.0002948917516983701, P160=0.00033044027579988715, P161=0.0003940348792147349, P162=0.00017379320363898688, P163=0.00014179258561313932, P164=0.00011084681470570154, P165=0.00012666505044707483, P166=0.00015451590902663273, P167=0.00018661401691110138, P168=0.0001325567207173751, P169=0.00015791014372917858, P170=0.00013437535367707322, P171=0.00010836211594259674, P172=7.205281020893115e-05, P173=0.0001069087678304534, P174=0.00011590918896545528, P175=0.00015008272012220246, P176=0.00017607465725696532, P177=0.00015977813124628428, P178=5.1098530869447595e-05, P179=2.3421071128968503e-06),Row( P0=-460.914722907967, P1=-0.9777852975236153, P2=-37.003763327266306, P3=-9.447491511926968, P4=-24.895698839424636, P5=-10.476853101442119, P6=-4.259308109061908, P7=9.256209866507643, P8=6.428888637503087, P9=0.8599535034213833, P10=2.6633423727727097, P11=6.485045668898187, P12=10.072859453790889, P13=13.345276500095975, P14=9.511489006172377, P15=2.510334774007299, P16=-11.043660203985933, P17=0.9692384194471855, P18=3.603193406327693, P19=4.682367833150734, P20=2.793320576296215, P21=-1.2676034452837737, P22=5.053460874221878, P23=11.202174752456813, P24=4.345353443646317, P25=4.7183979769309365, P26=3.5163083688960857, P27=2.298668784876069, P28=-2.4500955371396533, P29=5.11229521068077, P30=-1.0330540073112893, P31=0.5698981119157288, P32=1.9392355775878685, P33=4.374131117903183, P34=0.4851504826482956, P35=1.6264138921465894, P36=3.498450942514625, P37=3.2715240879854024, P38=-0.3435423123185182, P39=0.7644139176596065, P40=0.5491281647134096, P41=0.5987871627760711, P42=0.6112882530273717, P43=0.6112722450874972, P44=0.5736743176919054, P45=0.6197447368205582, P46=0.6673850667243939, P47=0.6000294996144234, P48=0.535313719390995, P49=0.5670872576266611, P50=0.5255762631504873, P51=0.4925220343613339, P52=0.007953648731600748, P53=0.000947126738342524, P54=0.00012085784832879671, P55=1.7850960239648433e-05, P56=1.0605756049471526e-05, P57=2.8594618419012994e-05, P58=3.3292726710650876e-05, P59=2.1810307503601453e-05, P60=2.3801502051780216e-05, P61=3.5203696592707834e-05, P62=6.172805751801635e-05, P63=7.492653206551698e-05, P64=6.182291074241422e-05, P65=0.00011051642102001381, P66=0.0001468240974196757, P67=0.00013979667851419543, P68=0.000314421523037775, P69=0.002374780675717417, P70=0.01161007977996684, P71=0.03133520997870656, P72=0.08260792312721289, P73=0.12101597302151063, P74=0.2073350259162312, P75=0.2519455501590437, P76=0.10006257663693775, P77=0.05058218268870188, P78=0.04112767371729813, P79=0.056295396101975745, P80=0.06986209633752404, P81=0.07502200673666011, P82=0.15844245285242756, P83=0.23589962236698064, P84=0.28295979508583985, P85=0.2820555597836834, P86=0.31813776636100943, P87=0.6998808816370669, P88=2.154272553231868, P89=2.8381968978132455, P90=2.1719605815382534, P91=0.8591247079739565, P92=2.2646223042772466, P93=3.9971533973364055, P94=7.668349696841769, P95=8.420885798117913, P96=0.8340402709107904, P97=0.20481225966810943, P98=0.2741337893388167, P99=0.24913764087202353, P100=0.06822615479568572, P101=0.021545358295865583, P102=0.007171280501064876, P103=0.007072241759976294, P104=0.015882641572165186, P105=0.06377078320624668, P106=0.7682510370015422, P107=1.556860262306287, P108=0.32520673961458124, P109=0.1611225613515197, P110=0.25046350378101667, P111=0.2764279862030579, P112=0.13508844270457088, P113=0.028568856467080267, P114=0.05561134793864924, P115=0.1923062599560404, P116=0.26125871820856156, P117=0.03484934399689684, P118=0.03921463807985959, P119=0.03417721532531653, P120=0.1996497668755798, P121=0.2105896947394458, P122=0.1914111107469687, P123=0.201055710828725, P124=0.05792579091305134, P125=0.04454566626692004, P126=0.18346023892217045, P127=0.1311755533231271, P128=0.06886874786140239, P129=0.02915990069178895, P130=0.05783716933253466, P131=0.0684109064951105, P132=0.03681305003197711, P133=0.031378824076365504, P134=0.015963974379156302, P135=0.005810840176420207, P136=0.028435926561719373, P137=0.10967559023611771, P138=0.04863968452778355, P139=0.023792117160355574, P140=0.05744693456015924, P141=0.06699686189547788, P142=0.055036690392890594, P143=0.06804785532737959, P144=0.0149067090801421, P145=0.00795534554420672, P146=0.007663530816725102, P147=0.011176017933022347, P148=0.00911962741358052, P149=0.022993879818288415, P150=0.019519111022803577, P151=0.007815560609037854, P152=0.012559161171926387, P153=0.0049337891186345285, P154=0.00807317810141512, P155=0.00984885994494654, P156=0.01688764642108733, P157=0.0066164468420444825, P158=0.0031669997701862062, P159=0.0021450354183392938, P160=0.0025460990708109153, P161=0.0011675312254791682, P162=0.0005944726333370952, P163=0.0004343248782478445, P164=0.0012057795506057436, P165=0.0019991781676328177, P166=0.0013648117191039787, P167=0.0009688635673235358, P168=0.0007075419231587675, P169=0.0006552867835650796, P170=0.0006666770930994202, P171=0.0006625370010827765, P172=0.000616997051507542, P173=0.0005560446617079202, P174=0.0006011158957954999, P175=0.0006153231316931296, P176=0.0005076833681806357, P177=0.00047751118599610314, P178=0.0002683477481611294, P179=2.5861277314148413e-05),Row( P0=-669.5060625687508, P1=70.64140231493928, P2=-4.139013682308819, P3=11.66723105549237, P4=-5.760931434209093, P5=3.9739134504372218, P6=-9.35409929563151, P7=5.237023170049241, P8=4.65135844021798, P9=1.3731295742200293, P10=-5.209678009818831, P11=1.1515114791917254, P12=0.5467482312407655, P13=-6.1817441038060865, P14=2.1025994886358492, P15=-2.8115119549229903, P16=-0.5989957815315835, P17=-0.0446187263482715, P18=-0.7035844900262215, P19=1.5567312828568172, P20=-2.965110443596878, P21=4.575120421293555, P22=11.820114280320587, P23=10.102993954954462, P24=12.904662736403134, P25=14.953017283829814, P26=7.2349232289552035, P27=5.817175684215088, P28=5.245789108560803, P29=1.9506309844514524, P30=3.0911435787875297, P31=2.7674903559597186, P32=1.9597303961748047, P33=1.9415023929099369, P34=0.3746122916000727, P35=0.3334813023326068, P36=-2.0800767597392054, P37=0.6657028327250756, P38=0.3461848349640034, P39=0.26583075771624726, P40=0.6775223145955943, P41=0.6208973265036872, P42=0.6300392454317624, P43=0.7523277146383038, P44=0.7935101443852065, P45=0.810279931819279, P46=0.7765069285798621, P47=0.6597322202043296, P48=0.6065568863360533, P49=0.5998666627689386, P50=0.6356918855517767, P51=0.6652798588260845, P52=0.013126393964261653, P53=0.000472034476101433, P54=5.174710170766868e-05, P55=1.7311933323228853e-05, P56=1.2272683929419351e-05, P57=1.677331781184075e-05, P58=6.760912598051548e-05, P59=0.0003821386691765997, P60=0.004384421817941486, P61=0.01920164732782831, P62=0.0396346805136843, P63=0.028934726205462134, P64=0.004692941573829362, P65=0.0014583552276417105, P66=0.0002088211662446265, P67=9.580409566179765e-05, P68=0.00034754699226189034, P69=0.002130683913751166, P70=0.0066460899773719585, P71=0.014722046081802437, P72=0.02317053553009988, P73=0.03766416958339217, P74=0.0511037921949686, P75=0.039258147802706964, P76=0.010736022824610525, P77=0.0016193916879640026, P78=0.0019239839686326287, P79=0.0031877906269233893, P80=0.005099101295960366, P81=0.014279515311820585, P82=0.022436050640927224, P83=0.018265140889733, P84=0.013283724223384273, P85=0.008958886447884644, P86=0.0053542456444954315, P87=0.0022420105019252384, P88=0.0024759983329003398, P89=0.005444287732915709, P90=0.01516660282321812, P91=0.013707591929757589, P92=0.011226368698990718, P93=0.007875326088410566, P94=0.011069452694467798, P95=0.012344232719860876, P96=0.00782212615846041, P97=0.013228878551640483, P98=0.006598565244136556, P99=0.0027402510277932265, P100=0.0014034005891530033, P101=0.0006206341743269745, P102=0.0005610629254265349, P103=0.0006468838855462231, P104=0.0005681396974827116, P105=0.000878339372367553, P106=0.0017698611883943452, P107=0.0025754272411758804, P108=0.001320576461083506, P109=0.001383595303005612, P110=0.003525436455425319, P111=0.01529677756089417, P112=0.00774861557068961, P113=0.001480302039366675, P114=0.0010263614970769134, P115=0.004080578663548922, P116=0.0014815944533378919, P117=0.0005465588429594504, P118=0.0005813266196527046, P119=0.0007121529272434883, P120=0.0010507930290824105, P121=0.00024614600311473973, P122=0.0004916399326572421, P123=0.0011079941857446505, P124=0.00041754520027818397, P125=8.797938956668069e-05, P126=0.00018475122273743945, P127=0.00014968162193657622, P128=5.941931134277192e-05, P129=3.0219659593513864e-05, P130=2.6859262261857517e-05, P131=3.246595495032705e-05, P132=3.7521169542652096e-05, P133=0.00010231713228322116, P134=0.00010389489897291135, P135=5.602599736877435e-05, P136=4.325780528498799e-05, P137=2.7430593169486375e-05, P138=1.4771577010047027e-05, P139=3.1808235034199925e-05, P140=5.50631168283622e-05, P141=8.10180784613388e-05, P142=7.945664795629526e-05, P143=0.00011708442883984132, P144=0.00015042652603293154, P145=0.0002411824991058051, P146=0.0002552798245090317, P147=0.00025225230233890517, P148=0.0004726681625662464, P149=0.00020028910297888842, P150=6.460369261794744e-05, P151=2.7390194349746037e-05, P152=3.736682185665488e-05, P153=6.230784891994145e-05, P154=8.43128375600688e-05, P155=5.6777363205087305e-05, P156=7.411448051263942e-05, P157=6.67166006017234e-05, P158=5.5798702459992725e-05, P159=5.892664401649523e-05, P160=5.5087818281961134e-05, P161=4.676575759608051e-05, P162=2.9306631985077955e-05, P163=3.3619115976790425e-05, P164=2.5409347912420983e-05, P165=1.829902546537809e-05, P166=1.2359012923789495e-05, P167=5.9412042221320705e-06, P168=6.4551189828459105e-06, P169=4.166809339544955e-06, P170=5.18595546109235e-06, P171=4.28356535832934e-06, P172=3.0876943933712634e-06, P173=3.750917731124933e-06, P174=3.2616376293258373e-06, P175=4.823981169761184e-06, P176=5.112099744859852e-06, P177=5.941842463349837e-06, P178=3.7476693171457224e-06, P179=3.9833733596786593e-07),Row( P0=-636.4301993709735, P1=70.78716216536559, P2=0.4697739624772222, P3=9.122666523139667, P4=-6.999176214302645, P5=-2.3018140756967456, P6=-10.920191606359337, P7=1.59909416445423, P8=0.7245200155833895, P9=-0.13439287197616245, P10=-1.974912521908339, P11=-1.1908063406215725, P12=1.9185376557337217, P13=-8.431836888014804, P14=1.8772702415842262, P15=0.6788981930918532, P16=2.018661308718824, P17=7.257310699319407, P18=5.682003194007615, P19=7.906197649376341, P20=3.8559139550257306, P21=8.756886874605241, P22=8.160012819273044, P23=6.920450637324173, P24=6.458301388765226, P25=7.34944656765115, P26=4.204197119723439, P27=6.45398542257568, P28=4.979132775864722, P29=4.546603859843746, P30=3.234360868663282, P31=3.792258872586055, P32=1.7285230586909504, P33=3.7216872857406287, P34=-1.265822858888391, P35=2.100454350336999, P36=0.06726062968526794, P37=1.829455492099547, P38=1.372203980344078, P39=0.8525054079208696, P40=0.6553185255431896, P41=0.6305156568080692, P42=0.6129421954329259, P43=0.6487289002784629, P44=0.6973814468796273, P45=0.704427163072238, P46=0.7194946848856, P47=0.7028876664733004, P48=0.6942500214760146, P49=0.7090871794953617, P50=0.6869797374423199, P51=0.6513075794296883, P52=0.0025966617833934487, P53=0.0005527254244333979, P54=0.00010538316709281212, P55=2.8190792671625393e-05, P56=4.195844411944483e-05, P57=0.00024142562714849495, P58=0.0011055215420747427, P59=0.0032226713407083355, P60=0.009693532380680273, P61=0.03185065295807162, P62=0.05644207216872268, P63=0.046033112996847245, P64=0.015710795938452767, P65=0.01438143435341313, P66=0.01734129474006344, P67=0.012823256882653195, P68=0.01363415409319026, P69=0.016054295103656198, P70=0.012803205677931747, P71=0.010982344663092683, P72=0.013567497528581454, P73=0.020892585739399283, P74=0.03999792543424404, P75=0.057081072446729296, P76=0.03557097660665855, P77=0.038268823512490634, P78=0.043544834448445244, P79=0.054072354453068364, P80=0.04690441229481157, P81=0.03186945949951908, P82=0.017308711638015624, P83=0.008557579877672654, P84=0.00995188255822612, P85=0.00914157182032312, P86=0.0055597170472132355, P87=0.003812534118809709, P88=0.0026467855114106686, P89=0.0023970011761061936, P90=0.0037589635257321355, P91=0.007067954034728872, P92=0.007120900019450001, P93=0.002581280606417142, P94=0.004077946792120405, P95=0.005462125233472596, P96=0.004148372816220271, P97=0.005666077288664431, P98=0.007935394946128452, P99=0.0101191727132236, P100=0.010576571553598068, P101=0.0034888516306783093, P102=0.0008665687575487163, P103=0.0011254602832135665, P104=0.001342361155778078, P105=0.0023403351264217344, P106=0.003802757207327824, P107=0.004590791403048774, P108=0.002448046110689197, P109=0.0019667385799226047, P110=0.0035861966734166938, P111=0.014059089087859484, P112=0.00878873830488609, P113=0.004543078973402716, P114=0.002911711327859974, P115=0.0008427242628620448, P116=0.0003921775541321177, P117=0.0005275396932478478, P118=0.0005272997151025729, P119=0.00022758492327037416, P120=0.00021585212335216525, P121=0.00016307865339621643, P122=0.00023626017360147238, P123=0.0006039362196038454, P124=0.00048287547101086493, P125=0.0001324887974985733, P126=0.00012370541187851583, P127=0.0001525206823160689, P128=0.00011789214948030655, P129=0.00010763400706354835, P130=7.749351402812918e-05, P131=5.6435685444176994e-05, P132=6.254586865755591e-05, P133=6.546730506795701e-05, P134=5.8170083030752216e-05, P135=8.970332876484719e-05, P136=0.00010958706594199007, P137=0.00012604158197549302, P138=0.0001859199282372459, P139=0.00015543153751475459, P140=0.00010608734687569805, P141=7.431320517168648e-05, P142=5.345212787748266e-05, P143=7.773303492805686e-05, P144=0.00010494556378237321, P145=9.065205833271839e-05, P146=4.6823800097587036e-05, P147=2.2724741555330157e-05, P148=1.7293956527737932e-05, P149=1.0491583243898092e-05, P150=1.192303840273813e-05, P151=1.4904233104677628e-05, P152=1.7746317057093588e-05, P153=2.662133659767827e-05, P154=3.842076411790359e-05, P155=3.453711987087937e-05, P156=5.8227968614574795e-05, P157=9.266668895237757e-05, P158=0.00010166323313372437, P159=0.00025904173794358374, P160=0.00011242462731541511, P161=0.00010065435831679692, P162=0.00013745611473099935, P163=8.786789899143824e-05, P164=4.7121427866953935e-05, P165=3.8672458082813244e-05, P166=1.881242295791614e-05, P167=2.6994765006998354e-05, P168=1.6087249618839095e-05, P169=8.873696732925681e-06, P170=7.244567732488113e-06, P171=1.0638295929204582e-05, P172=1.071653547241296e-05, P173=8.950389333359676e-06, P174=5.416839298708588e-06, P175=8.98272880360071e-06, P176=7.335017022260221e-06, P177=7.818046262784189e-06, P178=4.541653810518639e-06, P179=4.705539383067808e-07),Row( P0=-757.1426701304385, P1=86.35918702719667, P2=20.50472893795879, P3=16.773252469958678, P4=0.4816955883261974, P5=0.2682686204696035, P6=-1.681193930554771, P7=8.08305070109362, P8=3.689740728596585, P9=2.9549772537405943, P10=-3.598470280176326, P11=3.0819155059540333, P12=0.7610505930009331, P13=-1.9863876389678254, P14=-0.5918029882273063, P15=1.1563964763049996, P16=0.4676899630117269, P17=2.6460095361347875, P18=-2.198437743224301, P19=2.9683667425853324, P20=-2.8460259934206356, P21=-0.008841802718096323, P22=1.0309399950591294, P23=2.321992917676568, P24=-0.13622102098844022, P25=4.420347518062432, P26=0.6966479639261659, P27=5.264155995974116, P28=2.0395378822482235, P29=-0.663124104290673, P30=-0.4603020017225281, P31=1.627833416972993, P32=5.032639805674934, P33=7.7839149550695295, P34=6.792002404401086, P35=6.996952155344949, P36=2.842560809445801, P37=-0.08113628826949056, P38=-1.3610015848698336, P39=-2.533652595382563, P40=0.7310177676843795, P41=0.6496196434134439, P42=0.6163597133069949, P43=0.6494415440130922, P44=0.6425237964374768, P45=0.6347569851516862, P46=0.6763559872960931, P47=0.656060194851963, P48=0.6188559447420204, P49=0.6051603758149197, P50=0.7123067192131616, P51=0.7901935132523594, P52=5.750055472157524e-05, P53=8.441433794758415e-05, P54=3.2240430135122015e-05, P55=9.306880071895432e-06, P56=1.000587767293216e-05, P57=0.00022430914168417433, P58=0.002983831052286473, P59=0.005631860870384332, P60=0.0027413350356412314, P61=0.00422728829793624, P62=0.003222019250024493, P63=0.0038563558332980097, P64=0.0023721741058040503, P65=0.0012294337540426443, P66=0.005182416450566334, P67=0.006140038338075578, P68=0.0023838955889210417, P69=0.0010268132994858802, P70=0.0030523726539902676, P71=0.004576950030930125, P72=0.002313660679001276, P73=0.0015830592927669768, P74=0.004095376805113866, P75=0.005200760828690395, P76=0.005548160345722756, P77=0.002542292528044876, P78=0.0004105745000995743, P79=0.0006938824134998563, P80=0.0016308326554956792, P81=0.0029107445980418862, P82=0.0043117412121463005, P83=0.0015443972077245254, P84=0.0007199942069261065, P85=0.0005063234812163019, P86=7.701962406369149e-05, P87=0.00016372309212361926, P88=0.0014697612485113263, P89=0.0016753262516537866, P90=0.0011845474690200804, P91=0.00052512970551457, P92=9.821010196607357e-05, P93=0.00010792967267927947, P94=0.0004542379763030404, P95=0.001411829266071016, P96=0.0008140752526714439, P97=0.0004483016947916135, P98=0.0001871627910028716, P99=6.249243850982833e-05, P100=5.4832764024719806e-05, P101=0.00012664441351472423, P102=9.497839562537927e-05, P103=3.274344196485326e-05, P104=5.522360537119369e-06, P105=6.979254422042027e-05, P106=0.00018467443531501677, P107=0.00018499429029903092, P108=4.7001878868420923e-05, P109=4.119602781915113e-05, P110=7.77352965787707e-05, P111=0.00013722251049380995, P112=6.155983772858619e-05, P113=3.748797748835664e-05, P114=1.5449312866908146e-05, P115=4.691884521216287e-05, P116=3.2733698157203795e-05, P117=2.4978924453242622e-05, P118=9.1144992755376e-05, P119=6.692383474186779e-05, P120=1.4020331676523745e-05, P121=2.1554533757095723e-05, P122=8.797265466360487e-06, P123=5.0271663880880105e-06, P124=7.275768252410558e-06, P125=5.335709654311898e-06, P126=2.9527387889123707e-06, P127=5.721939398271295e-06, P128=7.685466581984695e-06, P129=2.858065571345618e-06, P130=2.2095814705632827e-06, P131=4.858786016917098e-06, P132=3.869925298014144e-06, P133=4.401833110507611e-06, P134=2.890414644951951e-06, P135=5.408331916102776e-06, P136=4.673916940060324e-06, P137=3.2117386728022835e-06, P138=2.296605692661911e-06, P139=3.943542744518698e-06, P140=9.167507516679353e-06, P141=7.1049121419688035e-06, P142=3.7212101747565303e-06, P143=3.992819860656334e-06, P144=3.6955949228315105e-06, P145=4.583577925794262e-06, P146=7.321391467526445e-06, P147=3.2903025448320748e-06, P148=2.6299903039859543e-06, P149=4.108289052580963e-06, P150=6.360029191569193e-07, P151=2.3082173817835347e-06, P152=6.302448504738399e-06, P153=5.457420614354059e-06, P154=5.986877581113973e-06, P155=1.0678082070502954e-05, P156=4.996956564613995e-06, P157=1.0258879987940829e-05, P158=7.176913346631375e-06, P159=3.4847386232473907e-06, P160=3.386931125761516e-06, P161=2.594495997857079e-06, P162=3.120741212928141e-06, P163=2.22273416574415e-06, P164=1.1700812070498082e-06, P165=1.4269919563920108e-06, P166=1.1553751939879495e-06, P167=7.192123727442346e-07, P168=5.466687147701011e-07, P169=6.604153253012003e-07, P170=7.233648171821123e-07, P171=6.847421111176188e-07, P172=5.657588206386237e-07, P173=4.562993798622395e-07, P174=5.603601629344744e-07, P175=8.705803135011826e-07, P176=7.427342242347588e-07, P177=5.87903689069575e-07, P178=2.2081052594721985e-07, P179=2.6502072624363005e-08),Row( P0=-700.5544628449862, P1=86.96604201562593, P2=8.322381619043627, P3=10.388355862360795, P4=-3.9685034738717335, P5=0.338118247775626, P6=-12.05876863432346, P7=6.382797276260603, P8=2.423743990731927, P9=-0.0444679166251115, P10=-5.265543452402736, P11=0.5070140544102489, P12=2.07672049128042, P13=-1.8050163280278155, P14=4.7440689114125, P15=2.684222927368181, P16=6.7667401955644255, P17=8.508343801332524, P18=3.2516207217019155, P19=7.484794341547564, P20=2.477723940887516, P21=3.522232307866063, P22=6.917336951882472, P23=5.649067186387275, P24=4.426803266673374, P25=5.07811601151695, P26=1.2199377647472964, P27=3.967391397274765, P28=4.74795308691376, P29=6.349096585725075, P30=3.1251380832964593, P31=1.210553886058862, P32=1.1107022338678312, P33=1.359288939068897, P34=2.9073733317685373, P35=2.229714030825524, P36=-0.37501568326195217, P37=-0.6562243853592097, P38=-0.2049776263250646, P39=0.43243251656549264, P40=0.7233453973564486, P41=0.678380591948268, P42=0.7076458311145162, P43=0.7135786759728484, P44=0.7057344250712617, P45=0.6951560786811019, P46=0.7327243589978082, P47=0.7041997349598946, P48=0.6306996521844223, P49=0.6423402673259073, P50=0.656826847174331, P51=0.7035982674219232, P52=0.0006551839636694476, P53=0.000603133354440144, P54=0.00018468886540316313, P55=4.477574418753086e-06, P56=1.0356974776549021e-05, P57=0.0001381556319031736, P58=0.0008419243809818553, P59=0.0024766779177798214, P60=0.0037399736852343035, P61=0.00448464134854163, P62=0.006254360811000163, P63=0.008334311729883786, P64=0.005230943944963309, P65=0.006105287416373516, P66=0.01355716901010304, P67=0.024456544234808687, P68=0.014712468017227908, P69=0.008315516299769218, P70=0.01013358520482088, P71=0.005992327336495754, P72=0.0026452183110104517, P73=0.0032016377940255825, P74=0.006014157933670952, P75=0.00940857710782896, P76=0.007525459313940845, P77=0.007593862389315578, P78=0.012181216934741662, P79=0.015009505265189136, P80=0.009795626245132251, P81=0.005428707379696999, P82=0.00610786369968776, P83=0.00976894418896711, P84=0.01172057848652178, P85=0.009219435969248548, P86=0.005487083780542664, P87=0.004493254738884563, P88=0.003996106760435457, P89=0.0017764201377823876, P90=0.0012353108977138335, P91=0.0021071263533178055, P92=0.00347623301309376, P93=0.0026789882069470122, P94=0.0022338019756401543, P95=0.005274325885781734, P96=0.0016302965279399605, P97=0.0010920096139869608, P98=0.0006231906175610999, P99=0.0003278858978967799, P100=0.00012550241655697706, P101=8.878594249016642e-05, P102=6.144903045782749e-05, P103=0.0001480744896406012, P104=0.0003495872797925115, P105=0.00017630697022182865, P106=0.00019420347563865351, P107=0.0001601222416741757, P108=0.00011137425205916389, P109=0.00015400260896362018, P110=0.0004106502788752203, P111=0.0016520332870916003, P112=0.0005042798345540347, P113=0.00048602006556414015, P114=0.00022227270314353277, P115=0.0002908740993730986, P116=0.0003440357313110041, P117=0.0007362235626983114, P118=0.0007269055976475283, P119=0.00035407805406263384, P120=0.00019285368273783325, P121=0.00024742837533040196, P122=0.0003839918193042673, P123=0.00033652512089172844, P124=0.00017304795480660535, P125=5.816527120632364e-05, P126=8.044411387508847e-05, P127=6.980658976689426e-05, P128=6.480402801140947e-05, P129=2.394939854921145e-05, P130=5.941603177335769e-05, P131=6.153671420482332e-05, P132=7.98375747070152e-05, P133=5.972704365157556e-05, P134=2.3201429590843682e-05, P135=1.410045437392505e-05, P136=7.55124217089163e-06, P137=4.332080185109644e-06, P138=3.188359730761676e-06, P139=5.8300137047508505e-06, P140=8.225926317469565e-06, P141=1.1101679004853892e-05, P142=1.6664865027864122e-05, P143=2.076114527040615e-05, P144=4.250188710503273e-05, P145=5.426780439972924e-05, P146=7.13540732706925e-05, P147=9.264247808240666e-05, P148=9.75044215569061e-05, P149=5.196252813384919e-05, P150=4.239199077884275e-05, P151=3.016368491534984e-05, P152=3.8803116885477325e-05, P153=2.775817887062602e-05, P154=4.349666692678259e-05, P155=5.829089313941106e-05, P156=7.027371197978287e-05, P157=7.109349445433259e-05, P158=3.818531532153807e-05, P159=6.779109235750375e-05, P160=0.00013260429813539436, P161=5.692357343486107e-05, P162=4.051758374022674e-05, P163=3.641991720088066e-05, P164=3.3065910359467e-05, P165=3.229465449776892e-05, P166=1.7789248200511158e-05, P167=7.764124230460924e-06, P168=5.4382539434328165e-06, P169=3.6417987609047927e-06, P170=3.075106368801611e-06, P171=4.371279529757625e-06, P172=4.405920188902654e-06, P173=2.281589373565492e-06, P174=1.867236168187508e-06, P175=1.716167987606577e-06, P176=2.445335667199212e-06, P177=2.6806359612886253e-06, P178=1.6870579999186724e-06, P179=1.656075813710645e-07),Row( P0=-713.4010906831458, P1=74.57173294920983, P2=15.694233297978931, P3=14.031621285186787, P4=-4.796249888847052, P5=-5.257474616696525, P6=-6.069648768933855, P7=8.101075440388897, P8=5.653202735011598, P9=5.877630469229735, P10=-3.705755171094422, P11=1.7201614667746832, P12=-2.6678058124121824, P13=-2.0124410090970097, P14=2.9995087388492077, P15=0.9016240251847301, P16=2.6812451174827827, P17=0.9756897768796282, P18=-2.683895446924901, P19=6.977728998183628, P20=-1.2385547592736155, P21=-5.052182195946824, P22=4.6141102573129364, P23=-1.9759491484520335, P24=-2.5861689393621647, P25=2.3959662730505364, P26=-3.0181294029683396, P27=3.9194510110665277, P28=-0.10688243162924967, P29=-0.20872908977945084, P30=-1.6386099510264047, P31=0.26284131093899304, P32=2.2601628454851106, P33=5.932389678725044, P34=6.62931521565914, P35=6.901957734621087, P36=7.708994823805117, P37=7.341290048295988, P38=2.6431060273667253, P39=-0.35979606358766264, P40=0.6087401007084776, P41=0.586997999093409, P42=0.5852329524436042, P43=0.5854382113424966, P44=0.5846034302445342, P45=0.6060700368758604, P46=0.5577703407874843, P47=0.5314999718900447, P48=0.5686089512052087, P49=0.6475870022509536, P50=0.7479456649051076, P51=0.6686340029972156, P52=9.32203686092123e-05, P53=0.00011532692770136623, P54=5.120374625555002e-05, P55=1.635859072780973e-05, P56=1.428017711108952e-05, P57=0.0007163636959362793, P58=0.00634975161974238, P59=0.009150343357163408, P60=0.0069930219357768, P61=0.006583642526481781, P62=0.0038412239306998405, P63=0.0020643914347939557, P64=0.002996340536392093, P65=0.011653365408694811, P66=0.023209451389445565, P67=0.012521177872206518, P68=0.008910706930927478, P69=0.01335003383101466, P70=0.01859708288157092, P71=0.010579055864693526, P72=0.003484234667344486, P73=0.013310502535528988, P74=0.012354559006076179, P75=0.003284253922427884, P76=0.0026588030387117213, P77=0.004065027168186086, P78=0.0043983083701661, P79=0.006856363911788854, P80=0.026990540864280734, P81=0.04050187694984814, P82=0.011670286165664902, P83=0.004149641848471335, P84=0.0036905340752181946, P85=0.0011511000821022967, P86=0.0007355065557659449, P87=0.0021787792977989583, P88=0.0019298598826689509, P89=0.0009679062813163937, P90=0.000978303504422356, P91=0.0003870259262915187, P92=8.095669584651515e-05, P93=0.00012229288325366918, P94=4.894731603330147e-05, P95=5.270338112609048e-05, P96=4.1019717027468434e-05, P97=2.582081117435166e-05, P98=5.9367830642344074e-05, P99=4.477613541356612e-05, P100=4.482277107585983e-05, P101=2.6638554465419085e-05, P102=2.5851448140417272e-05, P103=7.526610607416966e-06, P104=1.2455223583276949e-05, P105=0.00011842884516078513, P106=8.980495266787102e-05, P107=1.5762183988967022e-05, P108=1.9789884929708497e-05, P109=1.7206295478187478e-05, P110=4.272512505033509e-05, P111=0.0001696053136259344, P112=5.501715234399922e-05, P113=8.81016608599595e-06, P114=4.220940984397508e-05, P115=2.907252156058609e-05, P116=1.6112392521711224e-05, P117=4.8025547809662916e-05, P118=1.742541196301708e-05, P119=3.5247732948139305e-05, P120=2.1750538680557114e-05, P121=1.9981864577282387e-05, P122=1.3812459554207474e-05, P123=1.993289278130517e-05, P124=1.5716554428810215e-05, P125=9.906598801913036e-06, P126=5.213883236426581e-06, P127=7.852181968133155e-06, P128=1.0565610019265468e-05, P129=7.059722132139399e-06, P130=5.346705964899466e-06, P131=9.284251746418512e-06, P132=1.0604666356191499e-05, P133=8.485349922406635e-06, P134=1.1122148416775885e-05, P135=1.0239221433628495e-05, P136=3.5625365396467133e-06, P137=1.4419515108053115e-06, P138=1.5147916028649815e-06, P139=9.416660688576042e-07, P140=2.166521568298456e-06, P141=2.540369394769912e-06, P142=7.614495923585565e-06, P143=1.2229057568461199e-05, P144=2.1764733057129464e-05, P145=3.7457951086319e-05, P146=4.7436987845611014e-05, P147=3.088759693383388e-05, P148=3.4688769505092496e-05, P149=4.845852428024834e-05, P150=1.637781254797845e-05, P151=1.918784017863184e-05, P152=1.8039301052775707e-05, P153=2.2555555487837074e-05, P154=4.93843166735126e-05, P155=0.0001086234930224503, P156=0.00014986608230665684, P157=0.00021773226450405285, P158=0.00019510192154420365, P159=6.323168179163196e-05, P160=2.9076014766607893e-05, P161=6.917426766238408e-06, P162=1.238173258004285e-05, P163=3.138487992366826e-05, P164=5.108524887950844e-05, P165=4.5212839023968066e-05, P166=3.165508020385735e-05, P167=1.739989927049613e-05, P168=1.0858483351032728e-05, P169=9.079587642778978e-06, P170=1.0149560481333368e-05, P171=6.736314839208357e-06, P172=7.611836809730521e-06, P173=7.985392804281888e-06, P174=6.113084103258634e-06, P175=5.901561156783873e-06, P176=5.725742183364702e-06, P177=4.173145968004974e-06, P178=2.407128129308996e-06, P179=2.415043849320231e-07),Row( P0=-522.4271835924492, P1=40.9826368693461, P2=-14.093639221074064, P3=1.9425313429117712, P4=-14.932450513004236, P5=-9.298448252751637, P6=-12.467982809595421, P7=2.4331684055046128, P8=-0.5401430821986902, P9=-5.847455070325319, P10=-7.217408169990046, P11=1.0518000260732407, P12=3.383936281372654, P13=-0.5372017298431341, P14=13.005733874726383, P15=11.204071891924398, P16=10.67379326697065, P17=11.795081116233053, P18=5.91892972703909, P19=9.102304431265383, P20=5.172844874412498, P21=4.906660647475457, P22=5.590996973353784, P23=2.96894133328141, P24=1.8464282530617233, P25=7.289018270441126, P26=3.8483614202978287, P27=6.492996282508887, P28=3.32493580887729, P29=3.0629596167399344, P30=1.9031948541265054, P31=5.068077466424116, P32=0.3638183324071952, P33=3.687958319133658, P34=0.540029862887352, P35=5.317026090076492, P36=1.8489885982404137, P37=1.174334278939365, P38=1.5056603645084872, P39=0.8286061381707583, P40=0.6118223684186974, P41=0.5800838119481981, P42=0.5779429566174601, P43=0.6097214980240631, P44=0.6180363753273892, P45=0.5817397695324455, P46=0.6190328469878095, P47=0.642427026728526, P48=0.6074516921922368, P49=0.6466287060077949, P50=0.6908109428712972, P51=0.6304691461658588, P52=0.013943296440533695, P53=0.001034083191339566, P54=9.80850864487868e-05, P55=3.992227589315274e-05, P56=2.8908360481263416e-05, P57=9.261742154834269e-05, P58=0.0010272750396934692, P59=0.003131690478883372, P60=0.009115438963694128, P61=0.013015758101773639, P62=0.013029382380870829, P63=0.015783473750372437, P64=0.016563096637530292, P65=0.054025924706271745, P66=0.13682413967974305, P67=0.22109531799127274, P68=0.1540458506247611, P69=0.14004327330858388, P70=0.2003450724925818, P71=0.11991280454077782, P72=0.04663658377541992, P73=0.01962974301699675, P74=0.01295226821071316, P75=0.023098020047371555, P76=0.0235582542975306, P77=0.042504863649356114, P78=0.07867371048648765, P79=0.18570989013014474, P80=0.38378450732476627, P81=0.2312772466155099, P82=0.07549648792623091, P83=0.06472155531312526, P84=0.15743238670555335, P85=0.3092288633262106, P86=0.24632257177460926, P87=0.16940646920583158, P88=0.19059861849423554, P89=0.21966379609782305, P90=0.3837444142772214, P91=0.5054223285286078, P92=0.440774561074263, P93=0.1871744940940116, P94=0.12543201147113747, P95=0.12209820782128106, P96=0.056303591538749496, P97=0.06950248099248547, P98=0.05442245555019855, P99=0.02732842036354547, P100=0.027573065093978378, P101=0.05711760085095304, P102=0.07753558535295035, P103=0.1379657144192037, P104=0.03675754636741255, P105=0.009744112595295598, P106=0.01813275910695686, P107=0.03142041586171288, P108=0.0393610333147132, P109=0.01818069359054822, P110=0.009537399713400837, P111=0.05911004215393931, P112=0.0439234251898512, P113=0.047592634073893844, P114=0.05298019586382277, P115=0.02889967566657182, P116=0.03024549100346854, P117=0.033267352658386075, P118=0.04036981543376015, P119=0.0487806584789114, P120=0.008017700643004617, P121=0.005269792330093584, P122=0.008721844866576609, P123=0.017941385888448732, P124=0.013099145490426862, P125=0.009011752139902919, P126=0.01866279109041308, P127=0.020783771555810262, P128=0.005078702972697332, P129=0.006052238899496751, P130=0.002886937126160288, P131=0.0034916393580463104, P132=0.0024870836131332965, P133=0.0018591267905470276, P134=0.0015794825987393184, P135=0.0011366131065927093, P136=0.001196226432624111, P137=0.0019103374616856233, P138=0.001303396134214828, P139=0.00117022897646197, P140=0.001967375104964053, P141=0.0010571238804044595, P142=0.0015076786162750392, P143=0.0017640386610536512, P144=0.0016075478221980054, P145=0.002025090593303825, P146=0.0037873341547287596, P147=0.0008244274710204113, P148=0.0006979032875940568, P149=0.0005525877633020911, P150=0.000551250385105779, P151=0.0007331075617934002, P152=0.000828092278889127, P153=0.0014377941962044052, P154=0.0028849587065066583, P155=0.0032238667142524316, P156=0.002775753519330863, P157=0.0044532310517054995, P158=0.004164563918617882, P159=0.004243300878066567, P160=0.003978512661110979, P161=0.003340855501741293, P162=0.003846522052876188, P163=0.0033178685312276555, P164=0.0026220382016649787, P165=0.0016262775296405228, P166=0.0008566110470873021, P167=0.00042319372124263055, P168=0.000709437190073762, P169=0.0005225866569582701, P170=0.00047230417427812985, P171=0.00032684034001194265, P172=0.00029825880940602466, P173=0.000257815466217302, P174=0.00022733740281533497, P175=0.0002688770423536873, P176=0.0002922681994811705, P177=0.0002813324388208004, P178=0.00014926873979401927, P179=1.4926867592762238e-05),Row( P0=-520.3005301827817, P1=28.306706696490426, P2=-43.782100084048174, P3=-0.19021245128440264, P4=-23.540117908087886, P5=-17.95085834496415, P6=-12.804355619104347, P7=7.851655303716888, P8=2.827805136002868, P9=4.286136544026163, P10=2.772093949298423, P11=7.965498054797208, P12=15.376188638867744, P13=2.132883994543986, P14=20.82809712354767, P15=10.016911274789392, P16=-2.0010538024384092, P17=7.464598107540557, P18=-2.4254714483271065, P19=5.103691924341195, P20=3.5962241246258118, P21=1.1826341671416114, P22=2.1026288137339786, P23=2.3777608813648867, P24=0.12216940468819762, P25=7.695416028914764, P26=12.36421362099094, P27=9.01351741505069, P28=3.9854181029207756, P29=4.352240748588529, P30=-0.8567464111386534, P31=2.649600228274282, P32=-2.7593921355561943, P33=1.4486512534926377, P34=0.4307101563196525, P35=0.9530002223405631, P36=0.8143631349026217, P37=-1.6731967500792027, P38=0.07213079731293258, P39=2.214149338159921, P40=0.4536654996610327, P41=0.49378764467093533, P42=0.5772905830002986, P43=0.6294007814797968, P44=0.6074187667324977, P45=0.5130053822001259, P46=0.49621616396321805, P47=0.5441528779336291, P48=0.5547331905319254, P49=0.618352721333152, P50=0.6234633415052906, P51=0.5059441847352304, P52=0.0026330249337496946, P53=0.00031695620269201787, P54=6.435446605977133e-05, P55=4.870988230954683e-05, P56=2.625184517553802e-05, P57=1.1747549073356112e-05, P58=8.546941407958711e-06, P59=9.903860286099644e-06, P60=2.0003294363045984e-05, P61=3.702316694632679e-05, P62=7.508863519687573e-05, P63=8.281516212923124e-05, P64=4.083996825688897e-05, P65=8.220080671812334e-05, P66=0.00020471555030503148, P67=0.0004108414000009775, P68=0.0018720856640356985, P69=0.018206672799460234, P70=0.0635565965616799, P71=0.09232837587450825, P72=0.09273055274526329, P73=0.09333957708735618, P74=0.052067408927973485, P75=0.013078095057784471, P76=0.0126989745208718, P77=0.04212963153187307, P78=0.07607398126076728, P79=0.11218248291316356, P80=0.15455865837319602, P81=0.15952334330565004, P82=0.1634786290474842, P83=0.1741318315641097, P84=0.1492951147602227, P85=0.07899211315045458, P86=0.11823463796205054, P87=0.3770800911615376, P88=0.6950119527731707, P89=0.6496461930969782, P90=0.6839352489227413, P91=0.7036078870328664, P92=0.6326666150019018, P93=0.11515192914182544, P94=0.03897139674677829, P95=0.09767266176483838, P96=0.07736026295459349, P97=0.03301669744818657, P98=0.02438918743506619, P99=0.03830472512648203, P100=0.12451373172355543, P101=0.2509202172431224, P102=0.19605946505512387, P103=0.09878151228518771, P104=0.023436837125281728, P105=0.029437564385385785, P106=0.12279746883817247, P107=0.17660850054986194, P108=0.12411047953727458, P109=0.0843412934856038, P110=0.025168438180021593, P111=0.019010791435922228, P112=0.03047779993845592, P113=0.036516780013002986, P114=0.00778958150939869, P115=0.016625094898120255, P116=0.026847857176552727, P117=0.05948264900502213, P118=0.06294781913643065, P119=0.02943684294784096, P120=0.019159109465339125, P121=0.027513834399387066, P122=0.046515802026083, P123=0.08262150933861165, P124=0.0207206216169515, P125=0.0038071440710137476, P126=0.002991138384072398, P127=0.003157165378992282, P128=0.004126569275583145, P129=0.007167002373320026, P130=0.008216411682398094, P131=0.020228753099824793, P132=0.018441562672729412, P133=0.0024581871967681765, P134=0.005974950876716669, P135=0.004938129839874959, P136=0.009087097350053143, P137=0.009308902784085658, P138=0.005469199561065305, P139=0.006288835078599448, P140=0.00437642410162333, P141=0.005523870580898547, P142=0.010532723833399362, P143=0.0062141704020542265, P144=0.003288556585512175, P145=0.0009599099383430007, P146=0.0005706390901286057, P147=0.0003802081895101669, P148=0.00037518403066708045, P149=0.00040520928277565816, P150=0.00028399019062975105, P151=0.0002912011402757699, P152=0.0002588612657274104, P153=0.0002617363530394598, P154=0.00025806913896201533, P155=0.00034973532881703577, P156=0.0004977308657713247, P157=0.000978655647424124, P158=0.0018122724883680237, P159=0.0011536442407195065, P160=0.000739274812878298, P161=0.0003960223720818973, P162=0.00014976876281188594, P163=0.0001242439128618654, P164=9.78111318127769e-05, P165=0.0001283098670120181, P166=8.305612540788967e-05, P167=5.729984512812995e-05, P168=8.055334524574122e-05, P169=3.3731392239738844e-05, P170=1.9743471201224083e-05, P171=1.83461829073779e-05, P172=2.625683663534583e-05, P173=3.206675586675172e-05, P174=4.454550570966965e-05, P175=4.1009608263107634e-05, P176=3.025343193342808e-05, P177=5.196115069953975e-05, P178=3.799388809486127e-05, P179=5.5022200784243125e-06),Row( P0=-635.7999003466047, P1=46.9979753073262, P2=-8.721547109259905, P3=-4.970991732626213, P4=-17.282381294035286, P5=-2.349430620910701, P6=-12.979861865618462, P7=1.061164173750238, P8=-1.473610391347893, P9=-5.350522716689979, P10=-1.844327518298102, P11=-1.0459431795344125, P12=1.2930977780007684, P13=-5.978850352096928, P14=0.9147854676267038, P15=-2.639468328125172, P16=4.143181408805525, P17=5.605219712065787, P18=1.621495082625009, P19=6.57072060719885, P20=-1.6938212404989084, P21=2.7431367862072635, P22=3.561965128473819, P23=5.314644847661075, P24=2.147803347395884, P25=2.571543829682674, P26=2.6659108496173456, P27=2.7676719852507228, P28=1.9884798089905387, P29=4.046516313732907, P30=-1.608440701196764, P31=1.6425000995821208, P32=-2.7702188974006927, P33=2.4330458628890255, P34=1.3415770733394026, P35=1.9708157939343884, P36=-0.7435319123458313, P37=0.21963747683752602, P38=1.2738685312769793, P39=0.955890849428868, P40=0.6262296149618135, P41=0.5882689143744384, P42=0.6072722021252882, P43=0.6533393954716378, P44=0.6741930162103369, P45=0.6902073445533704, P46=0.7273189431291818, P47=0.7090166507682031, P48=0.6453923963290815, P49=0.6289618647872366, P50=0.6297521927312648, P51=0.6457830231416531, P52=4.4047139257690015e-05, P53=2.8098171332621157e-05, P54=1.4023367461354504e-05, P55=4.371004193693494e-06, P56=1.2365901910222382e-05, P57=5.945120649096825e-05, P58=0.0002465382138660676, P59=0.0006719727886223601, P60=0.0038752913308498734, P61=0.011478141953796787, P62=0.018513256237086273, P63=0.014693399169643054, P64=0.007507100767355687, P65=0.007421082140036931, P66=0.013841439143957175, P67=0.020647631600913992, P68=0.014499433361053025, P69=0.012210343201735469, P70=0.02325685143627026, P71=0.017019096112310825, P72=0.009365772128897073, P73=0.010944229319713401, P74=0.013532362258292924, P75=0.01601098718938687, P76=0.013538201608465684, P77=0.019937741303261633, P78=0.03682290068112059, P79=0.0722639280075352, P80=0.08119885824224411, P81=0.074902411951955, P82=0.07302678700118238, P83=0.05186153025759112, P84=0.06710610523259991, P85=0.059737026007991675, P86=0.010383691543546247, P87=0.006699439812690936, P88=0.010408339894306114, P89=0.011549922738293887, P90=0.02144585441644845, P91=0.019795466371436813, P92=0.037607058520389695, P93=0.02541015801622065, P94=0.029948603204374515, P95=0.054622738621688816, P96=0.03917277396849161, P97=0.028223971394402442, P98=0.01871184298900108, P99=0.014805676036309206, P100=0.005338343921246326, P101=0.006002774667093838, P102=0.003511813292573804, P103=0.0022613232163663823, P104=0.0012967876771230492, P105=0.0016272729803133068, P106=0.004473623026150319, P107=0.0034440342402945003, P108=0.0014728605028008122, P109=0.0011441134217028588, P110=0.0011853201758352061, P111=0.006914036996299215, P112=0.005918690788028442, P113=0.0032072356323089865, P114=0.0016694440409098941, P115=0.0008531504719127834, P116=0.000538570120078885, P117=0.0006571113558969268, P118=0.0012683300719435026, P119=0.0008201824283057815, P120=0.0009278651925260234, P121=0.0014724152736245461, P122=0.0006241757547077912, P123=0.00034643737884235636, P124=0.00037920899489033414, P125=0.00014731248620754406, P126=0.00018132832057520212, P127=0.00014883413729405403, P128=0.0001842821311582365, P129=0.00021660438194773078, P130=0.00011064041155907452, P131=0.00018756157419880695, P132=0.00011163610860815422, P133=0.00012197936403156823, P134=0.0001433210033931228, P135=7.154991098255566e-05, P136=9.356891845862358e-05, P137=0.00010197240276357155, P138=8.309697157999622e-05, P139=7.005327563390835e-05, P140=6.223586870779133e-05, P141=0.00013106141381419767, P142=0.0003105245854123739, P143=0.0002843370738091767, P144=0.0001526387635126133, P145=0.00011617127626174063, P146=0.00016641715707845547, P147=0.00012205098584904052, P148=0.00015362995941831204, P149=0.00025679564942441814, P150=0.00026670317941667794, P151=0.00032026583262732437, P152=0.00025272495169621626, P153=0.00022219296944084158, P154=0.0002723215910821432, P155=0.0002841792793231136, P156=0.00046249711961856026, P157=0.0004317344358595672, P158=0.000520903874920879, P159=0.0007258766921495484, P160=0.0006944429232761179, P161=0.00045580110849494207, P162=0.00034962259541218327, P163=0.00017677575592397175, P164=0.00015701848028226953, P165=0.00017128034891401305, P166=0.00012214981440384712, P167=0.00010170330896581211, P168=8.652181034924907e-05, P169=4.412677482858873e-05, P170=2.833954356076553e-05, P171=1.3276909268493474e-05, P172=1.724844193173327e-05, P173=2.252127810176439e-05, P174=2.5326236758559953e-05, P175=2.4933434382976505e-05, P176=1.733628739855853e-05, P177=1.3793138634067701e-05, P178=7.651699641599952e-06, P179=1.2472482851688287e-06),Row( P0=-668.9617856972333, P1=75.92889362779789, P2=-4.613420546318954, P3=1.358247780034084, P4=-7.205916598212422, P5=-5.038952142254797, P6=-9.247331316786166, P7=4.821781445353432, P8=0.826528300866411, P9=0.3798259423457621, P10=-4.510397575358248, P11=1.3327843225220624, P12=4.071347609272421, P13=-6.372594969560759, P14=0.3003273742554988, P15=-0.037201556087075184, P16=-1.2238093489876618, P17=-0.5711704935559703, P18=-1.8638353223305875, P19=-0.9170881026631635, P20=-3.989216216256695, P21=0.8699535017985912, P22=6.030913407278003, P23=7.52673895938887, P24=8.564693402742105, P25=11.06563441826274, P26=6.535365635534576, P27=7.207398712646693, P28=5.286163544057424, P29=1.6121824884483706, P30=3.423175521542633, P31=3.0842021867073477, P32=1.5207370040557233, P33=1.8701066902055257, P34=-1.4385474534988565, P35=0.4662725068347133, P36=-1.5624965930031198, P37=1.0913019132379127, P38=0.337334781500519, P39=-1.0028209624464195, P40=0.618619314241009, P41=0.5956203794566282, P42=0.6225741942194686, P43=0.6671298308475553, P44=0.7192588986507636, P45=0.6906894572123915, P46=0.6823907698918558, P47=0.6147602433148647, P48=0.5984452409447909, P49=0.6129670723755981, P50=0.6119301356926897, P51=0.6060293676737664, P52=0.00105853632453394, P53=0.00010499883660614833, P54=4.070987160383434e-05, P55=1.0195783842404539e-05, P56=4.8709860651451464e-05, P57=0.0005463862274089217, P58=0.0007728846515210311, P59=0.00191294810951145, P60=0.007919080179806345, P61=0.02411980005444519, P62=0.030607753549266578, P63=0.019423963055835998, P64=0.0031888010474856432, P65=0.0005582461889805847, P66=0.0008387251944184842, P67=0.0022386447265038876, P68=0.004074359200438233, P69=0.008639234456322477, P70=0.017403769117984424, P71=0.021671368249021855, P72=0.024665021290332864, P73=0.019833011156015573, P74=0.024977878519614256, P75=0.040623198403983214, P76=0.023915196595976677, P77=0.00941306754859707, P78=0.011698191643455879, P79=0.012683736004705149, P80=0.011018646517338693, P81=0.010561852235011198, P82=0.013426344243854428, P83=0.01301092891076839, P84=0.012946885268740257, P85=0.008397828162809977, P86=0.0043088132611853345, P87=0.0020836228755733554, P88=0.002632947745300196, P89=0.001552524104062506, P90=0.0019827616470515738, P91=0.004568042879153714, P92=0.005326803384271957, P93=0.002674468266006484, P94=0.0072172311916888585, P95=0.012388706778102507, P96=0.0015719721704448638, P97=0.0012319670697550752, P98=0.003562451801126152, P99=0.003676977862047218, P100=0.0024437592722214084, P101=0.001372651543443578, P102=0.0003301818158615801, P103=0.0001276259051669808, P104=0.00021010919322978806, P105=0.000592109900949194, P106=0.000741395079820028, P107=0.001042807298910168, P108=0.00042378087758246683, P109=0.0005693510267356815, P110=0.00023438580592969542, P111=0.0007431363166895, P112=0.0022323646106615857, P113=0.0019510100702012376, P114=0.0002630412517809441, P115=0.00012722312926536814, P116=0.00023984512532348808, P117=0.0002246901950452305, P118=0.0001393769533266363, P119=0.0001056007728773057, P120=0.00012103589120184605, P121=8.840016550382377e-05, P122=0.00015316453592931898, P123=0.00020457128306931833, P124=0.00015122907946965312, P125=3.384893684453889e-05, P126=3.262856017844616e-05, P127=0.00023396142344603066, P128=5.645140561295457e-05, P129=2.472786954031991e-05, P130=3.0385237732940133e-05, P131=3.477306995551128e-05, P132=3.126037407243886e-05, P133=3.810153308491543e-05, P134=3.645143873875488e-05, P135=4.0856995312357885e-05, P136=4.235093869214756e-05, P137=6.076478572764337e-05, P138=3.400850720538716e-05, P139=2.871342646671982e-05, P140=2.6735952119014975e-05, P141=1.4797809523191901e-05, P142=2.0046929357097028e-05, P143=1.392909360903241e-05, P144=1.827290826978807e-05, P145=1.7011587024107807e-05, P146=1.5337482644512842e-05, P147=1.3701652347807974e-05, P148=7.143366713591444e-06, P149=6.937785469900373e-06, P150=5.62948583142586e-06, P151=5.987642419342657e-06, P152=1.0602533394196097e-05, P153=2.4688351383195515e-05, P154=4.0300917501052005e-05, P155=6.679431162533613e-05, P156=5.614319942841079e-05, P157=5.190751195888607e-05, P158=4.4055634455267096e-05, P159=2.1340099393858973e-05, P160=3.752324050677735e-05, P161=4.098783516545191e-05, P162=3.078689024428166e-05, P163=1.5599076282451873e-05, P164=9.650642091303572e-06, P165=1.5204090671782545e-05, P166=1.3957405864724221e-05, P167=6.672506329904869e-06, P168=5.817953489799749e-06, P169=6.495763762777771e-06, P170=4.080801015091672e-06, P171=2.1342884095473517e-06, P172=2.3886791446469983e-06, P173=1.7485774510494145e-06, P174=3.1788701820100198e-06, P175=3.908488122030269e-06, P176=5.047337209222463e-06, P177=5.260584680713104e-06, P178=4.227526793524827e-06, P179=4.1862721264848655e-07),Row( P0=-555.2352053565943, P1=38.14177456629994, P2=-5.424091581286908, P3=-4.3579627061031205, P4=-18.771688383847135, P5=-11.306772146673417, P6=-12.300633287676542, P7=-2.9418488613280322, P8=2.397032162934092, P9=0.34624869931282065, P10=1.9048888188821114, P11=-0.13147239850414436, P12=5.882279927405673, P13=-2.6953526191144648, P14=-0.019110354936622623, P15=5.295591775345131, P16=2.4771176584937655, P17=12.275999654246467, P18=7.338075161562231, P19=10.13848932527406, P20=3.179141524341535, P21=5.939544015139176, P22=5.107100441010336, P23=5.616009448934361, P24=1.1690559202995472, P25=2.3075169968648597, P26=0.8523038796224707, P27=4.613592896726687, P28=1.7813525536936008, P29=3.9400730083082576, P30=3.015057844898885, P31=3.9398157289161264, P32=-1.1310551049129614, P33=3.784582181112954, P34=2.353945755997924, P35=1.6399303455536218, P36=-1.7045656589429612, P37=0.24399290588591932, P38=2.6270972478271326, P39=2.9857310936458465, P40=0.7061884683769198, P41=0.605572830647845, P42=0.5955603639692753, P43=0.6297277415987893, P44=0.6138181739893778, P45=0.6183762796767328, P46=0.6882208023629066, P47=0.7261012635753775, P48=0.7111162389782119, P49=0.6871250948630293, P50=0.6152611859014931, P51=0.6582331582462013, P52=0.0018669209249012777, P53=0.0001855851958469013, P54=6.0479127192485766e-05, P55=1.3486126282379262e-05, P56=2.387166784171517e-05, P57=2.754446298787971e-05, P58=4.1745926660292134e-05, P59=0.00014853746438549503, P60=0.00032967489280194697, P61=0.0008602364437062723, P62=0.0038278500579293005, P63=0.0072625411445267525, P64=0.008729230019850572, P65=0.013666303538464068, P66=0.0303338751588368, P67=0.06250484735285051, P68=0.027856518920029445, P69=0.005446071064112602, P70=0.005546815332460785, P71=0.01657851343731091, P72=0.031031662507713616, P73=0.024406914686108047, P74=0.022472120635976253, P75=0.038177884050411975, P76=0.06253639279231098, P77=0.30375649701122526, P78=0.9932689427842348, P79=1.155616972015802, P80=0.3376081242492672, P81=0.017465245143261518, P82=0.018664838815628356, P83=0.03465230091024899, P84=0.031611044207413445, P85=0.01492442382257055, P86=0.009261705410092563, P87=0.012133412622997598, P88=0.01857842227858714, P89=0.015879008782285632, P90=0.015965631921334145, P91=0.013371539202982714, P92=0.012092367333045452, P93=0.006650149332295947, P94=0.00866795662669775, P95=0.037832978467622735, P96=0.13134477195925512, P97=0.07440778491537045, P98=0.02597073280370063, P99=0.03508570667681863, P100=0.020307163997709547, P101=0.005094726550345028, P102=0.002122252704546584, P103=0.0030691085917718833, P104=0.0011418193271306117, P105=0.0029727778584387253, P106=0.005062020359601073, P107=0.002000159114373191, P108=0.0004666246602142628, P109=0.0004744754919725942, P110=0.0012845361536754482, P111=0.006694851304104443, P112=0.0034520033481828076, P113=0.000333488369301677, P114=0.00018788309151266915, P115=0.000252886185301305, P116=0.0007705617712042942, P117=0.009832248190646025, P118=0.0043930639871272706, P119=0.00035713332345676467, P120=0.000852891495659267, P121=0.002009763766956722, P122=0.00379499057377741, P123=0.00035296434979244576, P124=0.000213625336228388, P125=0.00030803262741595775, P126=0.0005015620436556125, P127=0.0006036322974901338, P128=0.00015149508659362935, P129=0.00015727853227447982, P130=0.00021975772741844113, P131=0.0001153375300656385, P132=0.0002890339473147574, P133=0.00042099402252338566, P134=0.0003482522012312716, P135=0.0006217062288730024, P136=0.0006729250124765525, P137=0.0005097384616696359, P138=0.0005866741862297933, P139=0.0007306222333653493, P140=0.0003878640007136087, P141=0.0005151964449340058, P142=0.00031904583362575627, P143=0.0002527898130531673, P144=0.00027666170941698396, P145=0.0003254630295674964, P146=0.00021628718210465187, P147=0.0002410811745408012, P148=0.0001369617836695689, P149=0.00020624449157844384, P150=0.00029379172131185845, P151=0.0005544996708377284, P152=0.0008595421334273305, P153=0.0010092183362440646, P154=0.000887313143438916, P155=0.0006696512589178582, P156=0.0008383100635630132, P157=0.000979299454911174, P158=0.0009351729563697925, P159=0.000898716347591357, P160=0.0010247121408691457, P161=0.0006412716348923479, P162=0.0005235499901181594, P163=0.00041689789171689315, P164=0.00025944398192893247, P165=0.0003165315626180388, P166=0.00023619693682293147, P167=0.000206434575606038, P168=0.00013112437179722169, P169=0.00012896881902080268, P170=0.00011397340406586233, P171=8.972837402091547e-05, P172=8.384005850579092e-05, P173=8.00438464459379e-05, P174=7.712434018364372e-05, P175=8.796910492009682e-05, P176=8.527152132638698e-05, P177=7.404508244334085e-05, P178=3.98329190327425e-05, P179=5.01619597382996e-06),Row( P0=-465.02135739493303, P1=-1.844224254336607, P2=-16.11302183114526, P3=-14.513368658664085, P4=-25.614822418211425, P5=0.4491800099898222, P6=-15.469485145177732, P7=0.296296348159444, P8=3.353846228623758, P9=-0.9440309948070595, P10=6.426857511327811, P11=9.813430042690015, P12=10.188984072190165, P13=7.803174739910849, P14=6.197338988593158, P15=6.392962854847078, P16=8.852606572136333, P17=6.8782704255368845, P18=3.226329927234285, P19=6.378711448211282, P20=-1.65894227181741, P21=9.598406617507726, P22=6.565559569304366, P23=4.104248442822005, P24=1.9343940174330245, P25=3.2477593072607904, P26=5.280649178711106, P27=8.177563446714917, P28=2.742487102418593, P29=2.334388744723641, P30=2.904836656096078, P31=5.678523916891719, P32=-1.3573566843698803, P33=4.6846742522018445, P34=2.886329831220599, P35=2.189955001057569, P36=-0.14535970506612972, P37=1.3955370888049938, P38=4.626383096913403, P39=3.6035118346141513, P40=0.6337257785861582, P41=0.6120302338228777, P42=0.6381405722952119, P43=0.6734010915038704, P44=0.6623014589566901, P45=0.6344537284055723, P46=0.6465030040511229, P47=0.6397017220681844, P48=0.6107604449689901, P49=0.5736751477681065, P50=0.5450547691128725, P51=0.5809471178606308, P52=0.04382659799368009, P53=0.0023721677007418245, P54=0.00010075110511166423, P55=5.211428451762979e-05, P56=0.0001912516155099538, P57=0.000722293812585397, P58=0.0004736442137181243, P59=0.0005204264628656048, P60=0.0007304271068670954, P61=0.0007815023909429283, P62=0.0025805203445491984, P63=0.004955110194777542, P64=0.005829834126054785, P65=0.015051864952110873, P66=0.018590059671483136, P67=0.02000917809601955, P68=0.04156489313833487, P69=0.08480802503349781, P70=0.20685968702721189, P71=0.17928978321703506, P72=0.21021758270607374, P73=0.3051340864489786, P74=0.1885074297812403, P75=0.1352846828102794, P76=0.11596342478101472, P77=0.1698593791475547, P78=0.34945131916926925, P79=0.5946108088764942, P80=0.2959524047416697, P81=0.04277510132787704, P82=0.041755162491694864, P83=0.09571547700732291, P84=0.16010332365929367, P85=0.15072698306946275, P86=0.09997863165112522, P87=0.09600757773182877, P88=0.13561655862494723, P89=0.18492787100445313, P90=0.16847342924216985, P91=0.27358715649742216, P92=0.4593603589977101, P93=0.47000200163378353, P94=0.5063741876375824, P95=0.6096587863229629, P96=0.912224268634453, P97=1.9845450487359102, P98=1.2649466553697335, P99=0.5688357613952961, P100=0.305321425950571, P101=0.1960537334403195, P102=0.13303688359356727, P103=0.14782635129487115, P104=0.13954865123160182, P105=0.28851563210356723, P106=0.4110412469485269, P107=0.27052349122068836, P108=0.1471206894927188, P109=0.11947653078688726, P110=0.14264355423271138, P111=0.17569274895778597, P112=0.10403524576848455, P113=0.045965977514441876, P114=0.02458044555307471, P115=0.02441361486786469, P116=0.041624993948689806, P117=0.13506875637070764, P118=0.05182134973857897, P119=0.02819645455790825, P120=0.04151896839730579, P121=0.018152258621418352, P122=0.01660305978700241, P123=0.05105791289762554, P124=0.032101553357285026, P125=0.017525781311860198, P126=0.029309538839804528, P127=0.06375515811438846, P128=0.08693618394288899, P129=0.010272443312609878, P130=0.007120441056499499, P131=0.00834582664819671, P132=0.008439411932349102, P133=0.010234824395471723, P134=0.007862385740301875, P135=0.01447074540815104, P136=0.014699641866438302, P137=0.015497941746001073, P138=0.00949273056400927, P139=0.023974193839611812, P140=0.007777590542239846, P141=0.005859740484061279, P142=0.01261047085607306, P143=0.03384060223470974, P144=0.06491789981424205, P145=0.03454915777852815, P146=0.055516535849863435, P147=0.057195085971100276, P148=0.032059312541557895, P149=0.02560291093787481, P150=0.021226164253099512, P151=0.015893611624626804, P152=0.022172264173071016, P153=0.023491262604911804, P154=0.028692473246090214, P155=0.024829510309556796, P156=0.03113336618022378, P157=0.034822125018390125, P158=0.033890365503059, P159=0.031877583396374205, P160=0.024683831044054003, P161=0.00963723091671717, P162=0.00804414027570211, P163=0.005244657144389405, P164=0.002800463535092142, P165=0.002309052115390462, P166=0.0029186681965054635, P167=0.0026422660972986187, P168=0.0018282292172741605, P169=0.0012518536616414034, P170=0.0013402954110575014, P171=0.0014326725282648668, P172=0.0014648377448499888, P173=0.0017629291056159735, P174=0.0016423117899950765, P175=0.0020220762853490983, P176=0.001486252508066785, P177=0.0011301559683676303, P178=0.0007216786079213245, P179=7.869524534697255e-05),Row( P0=-487.40280084309245, P1=42.73567373131786, P2=-6.542474125900299, P3=2.5433451647057193, P4=-14.501616588025984, P5=-6.758330814429548, P6=-5.47463500398229, P7=-3.2277111274157897, P8=-0.5623252721418661, P9=0.5068537370554178, P10=-3.1708807573567124, P11=2.796614390571121, P12=7.850173937695971, P13=0.3047441642840179, P14=6.696975055987401, P15=6.094069433375613, P16=4.85747906636594, P17=13.078381548511416, P18=3.448277373541287, P19=11.494873395323056, P20=7.646015501583003, P21=7.630860850680404, P22=6.74255639620007, P23=5.553010197907533, P24=2.436830011337356, P25=4.4093956368890925, P26=4.240051464001691, P27=5.534495176632095, P28=0.9250626910029465, P29=5.224842112923568, P30=1.7462239182645358, P31=3.9849949933086677, P32=0.08897867038903279, P33=3.2328128956081263, P34=1.21876742437703, P35=3.3351434188479065, P36=0.2360196438492745, P37=1.4095787346423905, P38=2.6583866431937406, P39=2.702799828148439, P40=0.6134859577974838, P41=0.5848014856072137, P42=0.5824217426368627, P43=0.6026857465142607, P44=0.6443733986383795, P45=0.6608875169848478, P46=0.7289679812049908, P47=0.7441348211790291, P48=0.6940784813478829, P49=0.6736529008877229, P50=0.6720365357997339, P51=0.6415776483511791, P52=0.04882258125212189, P53=0.06210034895616286, P54=0.048051778593414765, P55=0.001622239034919591, P56=0.0008289964549117204, P57=0.0013024622161842837, P58=0.0009201936962017971, P59=0.0012291860778841214, P60=0.0026111955153437743, P61=0.010866555648049853, P62=0.05597278292486783, P63=0.10131203053488853, P64=0.06090206359022367, P65=0.09940556730951838, P66=0.12543957776596934, P67=0.11530870149954253, P68=0.14631405109787124, P69=0.16273270719454777, P70=0.1646034275378678, P71=0.16514181457125163, P72=0.16107436474975811, P73=0.09365617208384804, P74=0.08063851894123322, P75=0.11390339750637372, P76=0.09574189539662704, P77=0.337150625953988, P78=0.5804779856547408, P79=0.6268843885527869, P80=0.45904705553703223, P81=0.20560106367292738, P82=0.15922513776045266, P83=0.1588811092049147, P84=0.1974451170257999, P85=0.22276098993837812, P86=0.21362489590803205, P87=0.20050494831854646, P88=0.27751471427985225, P89=0.2135109730794823, P90=0.25379494632527194, P91=0.38582667666067444, P92=0.4475552109834087, P93=0.3437364940248057, P94=0.47508676262049426, P95=0.4093169780243337, P96=0.2729448967817497, P97=0.24720571474513128, P98=0.14802548231186916, P99=0.08812151141468645, P100=0.05854301583621902, P101=0.07287605701996243, P102=0.05025803568530686, P103=0.129500970674996, P104=0.2998598855094038, P105=0.036868136268381084, P106=0.020740190077963315, P107=0.026137351086084722, P108=0.01617545674013117, P109=0.026243464900390145, P110=0.019762795093052003, P111=0.09358456888115015, P112=0.06022958152196778, P113=0.015010877910623168, P114=0.008643408815490437, P115=0.007176193699015388, P116=0.006622797512985784, P117=0.012077354469878065, P118=0.022017631796694834, P119=0.01890531882567155, P120=0.018839018729125592, P121=0.015875387034924148, P122=0.028286274302970395, P123=0.09697719945425991, P124=0.1001641505363093, P125=0.034976695738157114, P126=0.03414205033498317, P127=0.021313074250931992, P128=0.022655553070125116, P129=0.016366619530668418, P130=0.02940041318362033, P131=0.04021675889515056, P132=0.025976125170967868, P133=0.034051448723287386, P134=0.012935782483922253, P135=0.009695962184490033, P136=0.012115487803318406, P137=0.021863040869046724, P138=0.030060187714321486, P139=0.06540291309024512, P140=0.06871188754582386, P141=0.0639820722576884, P142=0.05580592094091032, P143=0.07051640182454272, P144=0.04672572498395623, P145=0.01419389911110131, P146=0.00825620258823197, P147=0.003624870081797666, P148=0.003788854829587245, P149=0.0031114273728523978, P150=0.0033606618597731524, P151=0.0026907026985104948, P152=0.0030548030182427515, P153=0.0027848137029479916, P154=0.002646798713355136, P155=0.0040661362114655, P156=0.006533652115305299, P157=0.008308723824735423, P158=0.011363922597591357, P159=0.01292242955282783, P160=0.010253100828827074, P161=0.00825251209531655, P162=0.004567873516165662, P163=0.0041488019997789035, P164=0.0038530568735342255, P165=0.00554953941970458, P166=0.0054689320102160035, P167=0.005047606849517962, P168=0.004204159494553191, P169=0.0036883547621257475, P170=0.002228907860532812, P171=0.0027843545693187056, P172=0.0033746609935754587, P173=0.0047317318575895535, P174=0.00483187753414994, P175=0.005048149779606533, P176=0.0052895268621997865, P177=0.003868887260861971, P178=0.0018904549773461597, P179=0.00020284118960281085),Row( P0=-630.6379923210338, P1=78.31821673497168, P2=0.8519439845421221, P3=0.30714811075222187, P4=-19.48499039547931, P5=-3.513266609415739, P6=-8.595064335106454, P7=2.2787350829409783, P8=1.4818545100262783, P9=-3.033963124239215, P10=-0.5243354100095741, P11=-5.840506593996286, P12=0.33445934277795375, P13=-7.115040830315047, P14=-0.3195644849870594, P15=-2.7003304034682243, P16=5.037561033545337, P17=3.871553802986971, P18=-1.653532199536238, P19=3.9975359524277163, P20=-2.0080746172657116, P21=-0.608459671368701, P22=-1.037398078781342, P23=2.03653601294264, P24=-1.1387005742166454, P25=3.150321280898128, P26=2.994301421357526, P27=3.201203009526509, P28=2.1516624318267663, P29=2.4683395029639357, P30=0.6060580517231748, P31=1.8784311574253334, P32=0.6173132186052722, P33=2.157293226621714, P34=-1.5344796810897152, P35=1.842263509379125, P36=-1.3680351511858762, P37=1.9720328540406349, P38=2.5442962581517223, P39=1.3245629909336225, P40=0.7220422586895633, P41=0.7136065256947318, P42=0.6712787071155348, P43=0.702021262552702, P44=0.7298776769739601, P45=0.709336853223115, P46=0.674213569483819, P47=0.6817049274510161, P48=0.694310761748362, P49=0.7290042749986602, P50=0.7169719006074063, P51=0.717314073660171, P52=7.995300132514103e-05, P53=0.0001610379258886632, P54=6.065982251472055e-05, P55=4.233172294433697e-05, P56=0.00018224713822388795, P57=0.0006670716144806622, P58=0.000712503329957433, P59=0.0016713946802580503, P60=0.00287554400714103, P61=0.004116418212038919, P62=0.0039947005469607, P63=0.0020939599621822747, P64=0.0018632304744878197, P65=0.0038214159857394864, P66=0.0037975787292894954, P67=0.006060684507325084, P68=0.009511219012373636, P69=0.009022704077827478, P70=0.010437911141435746, P71=0.010076292307191507, P72=0.012359442922439344, P73=0.008626454452137923, P74=0.003972067757227612, P75=0.003734903888079066, P76=0.0051703216798413935, P77=0.013216760850461627, P78=0.019543400827214303, P79=0.029306571604089802, P80=0.021021801760714963, P81=0.015896534135326882, P82=0.017473769383360278, P83=0.01725578141113716, P84=0.018519848975724514, P85=0.019732765720271748, P86=0.008185952242999183, P87=0.003090772557865154, P88=0.0027254847166759202, P89=0.0027811588574397582, P90=0.004891501484735398, P91=0.006372718665726592, P92=0.008074821714356811, P93=0.004964411955953781, P94=0.0032611467141363984, P95=0.003929045199995202, P96=0.0035568275473061785, P97=0.007409120679303548, P98=0.012487093978458035, P99=0.014323208587183186, P100=0.012905605597067712, P101=0.011448767941514861, P102=0.004985792593053758, P103=0.005201704941892726, P104=0.003809648204839292, P105=0.003400219920874032, P106=0.007258157504257427, P107=0.01691772007152105, P108=0.010868208963210776, P109=0.004139565250056363, P110=0.005329269925978906, P111=0.0318314845315198, P112=0.02242146699241946, P113=0.008039343812535888, P114=0.0016299573958103513, P115=0.002053012276952207, P116=0.0007194881483689154, P117=0.0006652074268336746, P118=0.0014726992498518396, P119=0.0004218545911861571, P120=0.0004158219524113025, P121=0.0004065034625696944, P122=0.0003826391945614621, P123=0.0005441139612619392, P124=0.000683915184544186, P125=0.0010399486199790637, P126=0.0007679367783205092, P127=0.0006763880723273484, P128=0.0004032410240654076, P129=0.00023963311249658098, P130=0.0004083284007565745, P131=0.0013172367874594712, P132=0.0013856349258550718, P133=0.0012677023764577548, P134=0.00154703091033679, P135=0.0011336941548616263, P136=0.001513029413168124, P137=0.0024572995656194072, P138=0.0022579472220003787, P139=0.0018871177234474315, P140=0.0013632680990904081, P141=0.0008352308072203979, P142=0.0005493261133367321, P143=0.0006536894928146909, P144=0.0006762875646665897, P145=0.0007387460428754615, P146=0.0002907680751258768, P147=0.00014891376987863676, P148=0.00012593029668034087, P149=0.00013806107985396025, P150=0.00011801695278666708, P151=0.00017891359640462466, P152=0.00026891470049842097, P153=0.0004308121956103026, P154=0.00042689790552401713, P155=0.0003533112079939194, P156=0.0003429133158423949, P157=0.00027882604925710624, P158=0.0001726701335609841, P159=0.00019873924236346791, P160=0.0001605044339693242, P161=0.00010611968175287822, P162=9.532333569742752e-05, P163=8.743038252544018e-05, P164=8.34363699326848e-05, P165=5.4896029329954715e-05, P166=3.10729947391695e-05, P167=1.8486982510152232e-05, P168=1.9523586737888987e-05, P169=7.74139388694389e-06, P170=4.65099785582358e-06, P171=5.053675132063318e-06, P172=6.530301256780995e-06, P173=6.417236533878952e-06, P174=6.5874975780460264e-06, P175=9.688413811597454e-06, P176=1.27897480253577e-05, P177=1.2757365102094462e-05, P178=9.051314611921286e-06, P179=1.134615356116954e-06),Row( P0=-496.90270517211195, P1=6.889835732682393, P2=-29.599405555833062, P3=-8.544920718891587, P4=-16.512824942370113, P5=-3.731559812813336, P6=-6.562428828797393, P7=8.48394253370122, P8=8.463795444933215, P9=5.397926203083843, P10=11.730380968280993, P11=10.277964912146953, P12=9.293446743853652, P13=7.056082684413152, P14=11.911026289420867, P15=1.6400646230400446, P16=2.0391468324120745, P17=3.593921818333975, P18=2.9639586606393484, P19=7.843857193880601, P20=4.805977126067622, P21=0.20595164077279457, P22=9.08460504598396, P23=8.050638167552068, P24=2.771966947585654, P25=3.922427641365959, P26=3.1183401022792223, P27=3.9923226002384844, P28=5.008885220490503, P29=3.8424648028224113, P30=-0.33505535573057416, P31=5.726557873630491, P32=1.3983094605252175, P33=2.531958129170848, P34=1.4218099640627715, P35=1.3991002524224616, P36=-0.225219123740187, P37=1.710931392717532, P38=0.853250107957675, P39=3.365904767410636, P40=0.5784931783806285, P41=0.6352298874254331, P42=0.6223539511692732, P43=0.6293390302724285, P44=0.6945373524625256, P45=0.635126896523152, P46=0.5830514840854755, P47=0.6305244163779818, P48=0.6081682016438235, P49=0.5860342227847782, P50=0.6118630486949169, P51=0.6131569314674082, P52=0.0066018138558640894, P53=0.0008029559531781328, P54=0.0001704292132828874, P55=9.904063265180247e-05, P56=8.338731744567915e-05, P57=7.397611030418142e-05, P58=4.95763606210649e-05, P59=3.7465001495927705e-05, P60=5.060266895334997e-05, P61=8.644632789563494e-05, P62=0.0001497059269037995, P63=0.00017809381860339055, P64=0.00013858129204770528, P65=0.00018354729187649695, P66=0.00019131325432532626, P67=0.00024141562517335726, P68=0.00045363874213463307, P69=0.002298978850162517, P70=0.015612137497658215, P71=0.055101886042769006, P72=0.04430538572709859, P73=0.043378359275858536, P74=0.1494342471364877, P75=0.37377532299223554, P76=0.2627156743028605, P77=0.16325513356116286, P78=0.3087845368019224, P79=0.23044752442865585, P80=0.044856531729007984, P81=0.011303460302396833, P82=0.01790180424363515, P83=0.06884065379191326, P84=0.2048610161747403, P85=0.4433291159442939, P86=0.2810374515877221, P87=0.16746337179983622, P88=0.32746485112824053, P89=0.35898576706493973, P90=0.39348949574338665, P91=0.48881623172446037, P92=0.5877133954808512, P93=0.6360982718850218, P94=0.7873295133219044, P95=0.39883189801341606, P96=0.43355805726897695, P97=0.5372095029474776, P98=0.810855529777079, P99=2.286634057689934, P100=0.44147288104026483, P101=0.049019523253520186, P102=0.053789833800872586, P103=0.05250831467874419, P104=0.03527629511964733, P105=0.21005257640723413, P106=1.0645822507314509, P107=0.7034116307655103, P108=0.2157070469933484, P109=0.056729261902622494, P110=0.08665331710662547, P111=1.043782259643407, P112=1.0252301737308194, P113=0.23220381863747147, P114=0.019865408472869252, P115=0.04513276180887129, P116=0.06867087221729606, P117=0.10199306760565761, P118=0.06024602803577198, P119=0.017163768564715534, P120=0.016593246119175595, P121=0.02233155061505538, P122=0.06746116399291693, P123=0.08445296387816606, P124=0.08486550890312936, P125=0.011518283816590992, P126=0.004674928220775559, P127=0.015171008395854505, P128=0.03213848740216974, P129=0.024797497728920623, P130=0.022380858055570177, P131=0.013300134934718861, P132=0.015787358925219954, P133=0.0243574793223639, P134=0.016809629132856635, P135=0.018177293720439563, P136=0.012480729803508391, P137=0.0025254083036257333, P138=0.002394804346669562, P139=0.006441125977320951, P140=0.019183743064254355, P141=0.026843870507284248, P142=0.0562285332845265, P143=0.03732386847736684, P144=0.05296511849322596, P145=0.03803606934153266, P146=0.07287386875936627, P147=0.017227765656870887, P148=0.016977869020815467, P149=0.01944167628339646, P150=0.009350211321940588, P151=0.004848498018188168, P152=0.006109063298821348, P153=0.007646220186176368, P154=0.005252983481440898, P155=0.006269355090177807, P156=0.012824762539381127, P157=0.013120337512221764, P158=0.012722669369233547, P159=0.01364017004547894, P160=0.007838550559023195, P161=0.005774001299314073, P162=0.004785013808179956, P163=0.0034077939749424086, P164=0.004467097427032229, P165=0.004067635397896919, P166=0.002753334022166598, P167=0.0016597384342810332, P168=0.0011197502862468072, P169=0.000798749715176343, P170=0.0006739593837518828, P171=0.0011027174280465036, P172=0.0007376702389627273, P173=0.0004769766042824428, P174=0.000633713452524956, P175=0.0014539654511221114, P176=0.0026324700206881925, P177=0.003348203821774442, P178=0.0016707748213766324, P179=0.00014570536799113392),Row( P0=-660.3897080872154, P1=60.2420847147986, P2=7.955642668544471, P3=7.56286200137913, P4=-9.569097238564794, P5=-3.6332862114181386, P6=-8.012289501351713, P7=5.103291473569855, P8=3.427300776100259, P9=-3.914154563430989, P10=-2.2262200543350716, P11=1.8719391522345636, P12=-1.8783471678376593, P13=-2.1829157639302172, P14=3.7385717960791816, P15=0.36516553629304116, P16=3.3659566269007737, P17=5.034128355780351, P18=-0.6873545140310816, P19=5.968599506490821, P20=1.401780955491927, P21=4.41824797862037, P22=9.36009970914394, P23=12.515219728735996, P24=8.14826308602542, P25=8.489227220796089, P26=7.098133780053965, P27=2.697346066524875, P28=2.159632765702009, P29=4.881497240749839, P30=-1.207756801013487, P31=6.001647765700602, P32=-0.22872346925650036, P33=0.9311564302698807, P34=-1.8108147871538478, P35=-1.0715632323309365, P36=-0.20292008609188522, P37=-0.11834270576749417, P38=0.8150707530726995, P39=-0.647331260946208, P40=0.7040934459155641, P41=0.7085170967073497, P42=0.6555655845909554, P43=0.6555250168457369, P44=0.7213699019243808, P45=0.7230383603706867, P46=0.7429291690342296, P47=0.673080690396233, P48=0.627058010402102, P49=0.6427595827078871, P50=0.672421689190455, P51=0.6641298453440696, P52=0.0002690978146142844, P53=0.0004633640654304451, P54=0.00014949253722326106, P55=7.939687397362038e-06, P56=1.623609071667034e-05, P57=0.00023632332645296802, P58=0.0008714247220014817, P59=0.0012922943673275339, P60=0.00432080878407923, P61=0.016434991460820624, P62=0.03529585882595236, P63=0.030014254384071216, P64=0.005553383556306221, P65=0.0013031889057838518, P66=0.0014613975182416136, P67=0.0012872474780819387, P68=0.0027098987577389767, P69=0.0044508247534910715, P70=0.010182671142365467, P71=0.01459243591158213, P72=0.027436491311395916, P73=0.035444831546889245, P74=0.049461634857246695, P75=0.06482237546707725, P76=0.02350819454382125, P77=0.007867508845957213, P78=0.008668359937783487, P79=0.015756869858397762, P80=0.020826104442541713, P81=0.014040080348468768, P82=0.013754730323821195, P83=0.026250151887469507, P84=0.03897806914053351, P85=0.04363011677335029, P86=0.027555296260834833, P87=0.012772112210239491, P88=0.003101995344679395, P89=0.0026486091635845458, P90=0.003219174962806648, P91=0.010008674188742673, P92=0.046649301065059066, P93=0.02471402388290194, P94=0.012912347119598908, P95=0.00533158746191312, P96=0.0014744348635715087, P97=0.001821389505319831, P98=0.010638814268700593, P99=0.02061124909179397, P100=0.002182383278786062, P101=0.0009606171610980701, P102=0.0007097582681332996, P103=0.0007073082253075204, P104=0.0017537397363779753, P105=0.0012129234633556226, P106=0.0010113618796246395, P107=0.002437043453148212, P108=0.0014260935355993969, P109=0.0029891386027781617, P110=0.0008610838353334398, P111=0.0020363600149710115, P112=0.0011331664694683749, P113=0.0009669185969987496, P114=0.0008088305644708601, P115=0.0012738260502051584, P116=0.0007926029207118469, P117=0.0019101305994536502, P118=0.001026302455882907, P119=0.0005170528304088011, P120=0.0007114905764282217, P121=0.0006163843057885481, P122=0.00033539951035114926, P123=0.0003611830293813721, P124=0.0002975947611331045, P125=0.00018501689223067308, P126=0.0001906487241925788, P127=0.00031750514766590304, P128=0.0001438153453170376, P129=0.00025193344299470724, P130=0.0002765736336926543, P131=0.0003925233729705656, P132=0.0001724179022110617, P133=4.182480149893922e-05, P134=2.9453682464625334e-05, P135=9.765947929229827e-06, P136=7.197249019505741e-06, P137=1.6759524044292976e-05, P138=2.1241547843169245e-05, P139=2.195227437584869e-05, P140=2.7368683350007866e-05, P141=4.451114499522639e-05, P142=8.766597520624637e-05, P143=0.0001635277014768779, P144=0.00013998461741017326, P145=9.806087534599319e-05, P146=0.00015201937013942173, P147=8.5986611613691e-05, P148=0.00020398168958599104, P149=0.0004619028140520768, P150=0.0006180634446747577, P151=0.0007146699360420329, P152=0.0003693835636982679, P153=0.00016516123502166021, P154=0.00018284756428562913, P155=0.00022115222923402362, P156=0.00028929792416600554, P157=0.0003662013042878169, P158=0.00039474463270091953, P159=0.00044669689454160903, P160=0.0003951180191086749, P161=0.0002601198844807323, P162=0.00015509800913273726, P163=7.879984428998756e-05, P164=8.254298425715305e-05, P165=8.905895013724869e-05, P166=0.0001073484311948515, P167=0.00012160336469360188, P168=7.806316024557597e-05, P169=5.5202082872803466e-05, P170=2.7435356757562407e-05, P171=2.2477440934996274e-05, P172=2.5781325929939965e-05, P173=2.5149954077621695e-05, P174=2.3332138529409623e-05, P175=1.9391493383784818e-05, P176=2.1506781555953148e-05, P177=1.831627132255411e-05, P178=8.375994658186207e-06, P179=8.972363150156602e-07),Row( P0=-458.08811852880496, P1=11.169418901498942, P2=-32.95622265537247, P3=-3.9114338107120896, P4=-20.50948632802252, P5=-8.19380741556859, P6=-6.384924360443442, P7=13.180026193132496, P8=14.116999265280768, P9=6.849941429839946, P10=14.777415403541996, P11=15.627436341238692, P12=10.447157259437287, P13=3.3029839969678894, P14=9.085157556631453, P15=2.8819594002052957, P16=4.348683895025885, P17=10.974281784741677, P18=6.792241590061444, P19=11.140274374587076, P20=0.9410015204430032, P21=3.539876562424779, P22=10.403831853429224, P23=11.471375782106524, P24=2.192407113620249, P25=1.0943682022093382, P26=4.653639345178328, P27=4.833786687726234, P28=4.722345231355229, P29=5.657093972661077, P30=-0.38844962529546007, P31=6.514873165131345, P32=3.1896170591933712, P33=2.6918132111765543, P34=2.5290697608866086, P35=4.461014674676019, P36=0.012543970309792138, P37=1.267421279040725, P38=2.3654212103208967, P39=2.0078350979120656, P40=0.5684183331030611, P41=0.545452841962975, P42=0.5588451224648503, P43=0.5297512630868393, P44=0.48796723233595307, P45=0.5055973230836092, P46=0.6036572323111846, P47=0.7107237232445788, P48=0.6878798600352484, P49=0.6909869693056758, P50=0.6409017444297097, P51=0.5713367036392389, P52=0.10357354514465808, P53=0.016785067121315667, P54=0.0020095154482709958, P55=0.00017833748444894308, P56=7.61536692754586e-05, P57=6.0092472046664486e-05, P58=4.3270404770655725e-05, P59=3.1822395096136507e-05, P60=6.743327035340736e-05, P61=0.00014082401963305262, P62=0.0002053416386927813, P63=0.00021050687457271298, P64=0.00015676456517054406, P65=0.00021342227872682223, P66=0.0003569156718311655, P67=0.0005849765686707037, P68=0.0011608472881820678, P69=0.004055379339008254, P70=0.011058030194380335, P71=0.01356359134405279, P72=0.021404730977580652, P73=0.06729503318542662, P74=0.13416163615871932, P75=0.19967537851918016, P76=0.1392959753521544, P77=0.2573853944350677, P78=0.45492349815859845, P79=0.42726174773348274, P80=0.24740697792019867, P81=0.19680157953114036, P82=0.18875064943473135, P83=0.1963801483421763, P84=0.2854542687203858, P85=0.4083123361449671, P86=0.5518342098534954, P87=0.6466664857330783, P88=1.3140687856403785, P89=1.2644940666935947, P90=0.5217113331342773, P91=0.19883307133270606, P92=0.5921470884375181, P93=0.8395013529548789, P94=0.5023422548607592, P95=0.5886538770805636, P96=2.30544532550939, P97=3.9255183541020795, P98=1.6533069538402274, P99=1.1743703778989625, P100=0.5921594747743185, P101=0.09646482430263786, P102=0.02361811060007322, P103=0.046379577516126605, P104=0.046619604316992816, P105=0.03338601647153202, P106=0.08064475042036677, P107=0.18654601414285288, P108=0.281349792345772, P109=0.4016088863480685, P110=0.18836513864791854, P111=0.5175967771950721, P112=0.14905440247016757, P113=0.034583472877046155, P114=0.05950716873933296, P115=0.06953244721146894, P116=0.07023892255738304, P117=0.16099394036272435, P118=0.18674748396086593, P119=0.19993214758044714, P120=0.06433515478018607, P121=0.028052117017636113, P122=0.06572896705055732, P123=0.07795371507574822, P124=0.025980026087368442, P125=0.03090159988686995, P126=0.053666214244391344, P127=0.053490457094266454, P128=0.010145112906844661, P129=0.0096105958043648, P130=0.01569510574712155, P131=0.023276167718045596, P132=0.021246440642358633, P133=0.01911552586969307, P134=0.025379237319854628, P135=0.04013649233372186, P136=0.0064837791919610845, P137=0.008592429493774738, P138=0.020461784679001987, P139=0.028987079563484935, P140=0.013152047289917488, P141=0.014122427483390437, P142=0.03625986599335348, P143=0.04900193342198772, P144=0.024039605246996235, P145=0.07398301916530621, P146=0.31208178813420734, P147=0.10320921178554476, P148=0.12748375474363102, P149=0.04846260139829627, P150=0.009327965588488456, P151=0.02644573870073028, P152=0.01844701762618248, P153=0.008396471481109058, P154=0.014201865009539927, P155=0.005595347635621657, P156=0.012989393929370468, P157=0.013416014378835007, P158=0.027123804027944416, P159=0.026819428858407872, P160=0.014005477120896751, P161=0.013631383945012608, P162=0.0063401097424187315, P163=0.0033884268078805146, P164=0.0027093444287324756, P165=0.0038720811031229376, P166=0.0030367622484832875, P167=0.0028483513885304733, P168=0.0031233406497167293, P169=0.0024527362379580075, P170=0.0015708255376103782, P171=0.0016322788482982223, P172=0.0015735436865646417, P173=0.001816198916698053, P174=0.0019972322261066432, P175=0.001816358298107028, P176=0.0016947603447662195, P177=0.0014489816655597835, P178=0.0010945321568938942, P179=0.00010603556862454212),Row( P0=-768.1687193749997, P1=79.36445471677784, P2=26.263419061396267, P3=20.32501060970014, P4=-0.2431054382711007, P5=4.902934342941439, P6=-1.0296607186164508, P7=10.077317796437974, P8=4.950235632650595, P9=1.0713627370505872, P10=-2.189237606759828, P11=3.58318664410822, P12=2.066981738659042, P13=-3.5308250918315807, P14=-0.22557969985992565, P15=1.6197034007231421, P16=2.837644961286612, P17=3.2386530707916594, P18=0.4013746572595331, P19=3.5444536102730795, P20=0.5529237195333064, P21=0.537191783943819, P22=2.694965062026395, P23=1.9102973943286399, P24=-4.044157199685641, P25=2.2789555528825924, P26=-2.8169002637780136, P27=4.127548857980015, P28=-0.05223998858106351, P29=-0.6084440698181578, P30=-2.8439602161810624, P31=1.0834865194379217, P32=-0.8215538317244987, P33=0.2967413029848816, P34=-2.6280271235200106, P35=-0.7056356160996532, P36=1.029419445036489, P37=5.128312222805689, P38=3.420063094858331, P39=4.109197651442357, P40=0.6007143216091454, P41=0.6382190570970664, P42=0.6365360267969811, P43=0.6551724820426416, P44=0.6812029189255154, P45=0.7044099660370974, P46=0.7406416171390156, P47=0.7386353766154637, P48=0.7491536279492815, P49=0.7681931452138099, P50=0.7118838733781443, P51=0.6235918747827952, P52=4.470966017694204e-05, P53=0.00011199367426510155, P54=4.550445587611164e-05, P55=9.241720426681681e-06, P56=0.000683778115649609, P57=0.005977563934251128, P58=0.005676204265448986, P59=0.002458393855805176, P60=0.0016457004172651158, P61=0.0008566908137648973, P62=0.0009853235498813735, P63=0.002009865000102125, P64=0.003202605248498619, P65=0.009953322533636944, P66=0.00856727657940912, P67=0.002694509851832587, P68=0.0013910789905113813, P69=0.0026459468973994457, P70=0.002974592787587572, P71=0.001434075891044186, P72=0.001525222496154189, P73=0.002980822730932469, P74=0.0026981989461715475, P75=0.0012794508441811782, P76=0.0008566624866199964, P77=0.0016000053834876228, P78=0.0027079601505448168, P79=0.0037538910924763552, P80=0.0032511508788038465, P81=0.001824809290823705, P82=0.0007703294199534497, P83=0.0002699341382353042, P84=0.0002331404201606467, P85=0.00037323231854075, P86=0.000328994300846254, P87=0.0002065921402377071, P88=0.00022016604839720177, P89=0.000172082675735734, P90=0.0003210585763743272, P91=0.0005473234080464143, P92=0.0008330152126310666, P93=0.00046324299082709853, P94=2.733604754853582e-05, P95=3.214798780516076e-05, P96=2.7883220523917338e-05, P97=2.3077880155304372e-05, P98=5.478892678945066e-05, P99=0.00012184848346710034, P100=0.00018834604941024452, P101=8.774727243280118e-05, P102=1.427216548404322e-05, P103=1.4144664708981818e-06, P104=2.52098349678718e-06, P105=2.68110525612242e-06, P106=4.800816325569212e-06, P107=1.37582852014323e-05, P108=5.574576649952053e-06, P109=1.1919725150722004e-06, P110=1.2624051356517868e-06, P111=1.2673379229577996e-05, P112=1.6030184127713555e-05, P113=8.98751617004144e-06, P114=5.080858381849631e-06, P115=7.138114618017594e-06, P116=4.7534465270525304e-06, P117=1.2316311508254501e-05, P118=1.7717048644619196e-05, P119=8.958138924041963e-06, P120=7.673825016873846e-06, P121=1.486123048060425e-05, P122=1.3114667610355824e-05, P123=3.875956672487423e-05, P124=4.526159773442432e-05, P125=8.560035449552442e-06, P126=4.531539949656331e-06, P127=6.1809466233160015e-06, P128=6.3760350088203586e-06, P129=2.0048274332661565e-06, P130=2.28636136402624e-06, P131=6.722401670298941e-06, P132=5.033092668705069e-06, P133=9.05419720135426e-06, P134=8.502039856934648e-06, P135=6.8145988049709685e-06, P136=3.3625296933072888e-06, P137=3.1582389198458436e-06, P138=2.355753023162226e-06, P139=1.7895894354360624e-06, P140=4.145892151065469e-06, P141=6.33281862288252e-06, P142=1.12801331409449e-05, P143=1.5498841678916437e-05, P144=2.0249543661971316e-05, P145=2.4222445681511762e-05, P146=1.7397825809932535e-05, P147=1.8448381184020184e-05, P148=1.0025823047130295e-05, P149=1.8737018532038314e-05, P150=2.9006325225338762e-05, P151=4.2433014288020316e-05, P152=5.3214557568907666e-05, P153=0.00012555200550218583, P154=0.00011743530986730672, P155=0.00021000670422422507, P156=0.00020906543238192308, P157=0.0003076133137236457, P158=0.00010619271952988046, P159=3.67533562886533e-05, P160=1.795713252840862e-05, P161=1.7691612412713562e-05, P162=1.5105253688309595e-05, P163=1.7000197229828603e-05, P164=1.581219506235296e-05, P165=1.3200049996557345e-05, P166=6.751682163231842e-06, P167=6.2359976329133175e-06, P168=5.21207053079982e-06, P169=3.6255306708259214e-06, P170=4.343474080129016e-06, P171=4.783421760871842e-06, P172=3.4052709341849455e-06, P173=5.541323326813917e-06, P174=6.273622633190765e-06, P175=7.035286811423714e-06, P176=7.668350306814081e-06, P177=5.938416881966347e-06, P178=2.408144957204465e-06, P179=2.405554479773058e-07),Row( P0=-629.1637505992524, P1=73.76950964032491, P2=-12.462150801839307, P3=-3.0652276843961554, P4=-12.89272019073687, P5=-5.120356163649797, P6=-10.812369165545796, P7=1.5075125455642597, P8=2.252060194141332, P9=-7.87566541627786, P10=-1.23622330325826, P11=-1.6759309336174124, P12=1.4554159757220722, P13=-3.4973838425291497, P14=3.3425152975417047, P15=-2.7951864192920572, P16=5.701127541628558, P17=4.057029284543935, P18=-1.2250755952884311, P19=5.474561717279133, P20=0.6961006503533037, P21=4.526614039306748, P22=2.2102420042105835, P23=4.8837692947449325, P24=-0.8203216254217368, P25=0.36497353563075546, P26=1.7120107236617952, P27=2.9850641162894247, P28=2.521770956392262, P29=5.357618219051862, P30=0.1482226481251177, P31=4.744882881461059, P32=-0.48270076866782563, P33=4.458401869487891, P34=0.6319933796499239, P35=1.8852736193345694, P36=-0.4398487908750441, P37=2.6213187425331923, P38=0.5491154455110194, P39=1.6006051522700946, P40=0.7063709273450437, P41=0.6873552887432437, P42=0.6760378390960372, P43=0.6835335846089041, P44=0.6900949346703921, P45=0.7010677291060557, P46=0.6941349830849631, P47=0.7201138390802453, P48=0.7156067728462893, P49=0.7196907622747081, P50=0.6984507684320099, P51=0.69748627319135, P52=4.501931135330618e-05, P53=7.024097891843109e-05, P54=4.066584734467923e-05, P55=5.892017396982651e-06, P56=3.635932946203853e-05, P57=0.000168626873581354, P58=0.0007235513824777164, P59=0.0015607268304727747, P60=0.0015457352263255782, P61=0.003011512531874111, P62=0.006173586388933281, P63=0.005829306980641317, P64=0.0030557618388974512, P65=0.0034706332348769654, P66=0.007879486319448119, P67=0.010031278707463778, P68=0.007807284455115975, P69=0.013423442840022063, P70=0.018635436625815845, P71=0.010000682153379807, P72=0.007167006349813704, P73=0.009071329770196585, P74=0.01144510582636218, P75=0.015255708501025327, P76=0.01974445661214676, P77=0.03891454922451692, P78=0.03281234947569669, P79=0.029449290082961533, P80=0.02103496679742252, P81=0.013750397312518425, P82=0.01287018473698529, P83=0.008818687946108595, P84=0.006961151983581647, P85=0.006458553532955782, P86=0.005569062661432032, P87=0.0056671668959208665, P88=0.00694976260032581, P89=0.004802838042733211, P90=0.003711223341019628, P91=0.003648005934399586, P92=0.005029080632638972, P93=0.003145788763674939, P94=0.0026154610236606567, P95=0.003687458322255547, P96=0.004042798044764653, P97=0.008170410216460143, P98=0.006979490091955096, P99=0.003967578130374636, P100=0.0023060373877724074, P101=0.001972754785261345, P102=0.00246298887279897, P103=0.0014794844811946053, P104=0.0006979549183302129, P105=0.0006235599640660738, P106=0.0006954691311579536, P107=0.0014266078019635124, P108=0.0015022202873144212, P109=0.0013174628236381654, P110=0.0005526145914821551, P111=0.003215854824952046, P112=0.00310811707321447, P113=0.0022636341200988424, P114=0.0008277062418493414, P115=0.0006471004655907017, P116=0.0007849450745665951, P117=0.0008934793246524768, P118=0.000690605500738011, P119=0.0009977871541115503, P120=0.0017998553787565619, P121=0.003065188623368033, P122=0.0027319390640093253, P123=0.0018125818436038048, P124=0.001043903909482582, P125=0.0004689016563557615, P126=0.0004847924859782062, P127=0.0004707303709309768, P128=0.0005413569759460693, P129=0.0003780916061277737, P130=0.0002655014307091824, P131=0.0001891135086213521, P132=0.00014718410707666187, P133=0.00013770783960498166, P134=0.00016257915209540912, P135=0.00023651719494839087, P136=0.00015780024838095273, P137=0.00012262758959321105, P138=7.99604090718957e-05, P139=6.451574623746236e-05, P140=6.15280174399742e-05, P141=8.614702116410726e-05, P142=0.00013911402915351627, P143=0.00011487715039802428, P144=0.00010238814577152489, P145=7.375837285905341e-05, P146=7.722896933937034e-05, P147=8.346639585952264e-05, P148=5.347833788224204e-05, P149=6.619660269181724e-05, P150=5.9614490038518656e-05, P151=6.895253221672326e-05, P152=7.120553512894049e-05, P153=5.732383596598082e-05, P154=4.33066919440884e-05, P155=4.229524103078723e-05, P156=5.416498466792465e-05, P157=6.844865577069409e-05, P158=7.232311264372419e-05, P159=7.82144043066709e-05, P160=0.0001001342648772506, P161=0.000110834792962866, P162=0.00012876874417218378, P163=9.045982924800835e-05, P164=6.032328800157795e-05, P165=5.192680878598191e-05, P166=2.8238025543947135e-05, P167=3.0807534802308414e-05, P168=2.553186302630669e-05, P169=2.4063433034678438e-05, P170=1.3764416735505649e-05, P171=1.0182853065718745e-05, P172=1.0133310841227718e-05, P173=8.933319498882536e-06, P174=1.1489644772191264e-05, P175=1.0573765389915115e-05, P176=1.2199208694522045e-05, P177=1.110837514872742e-05, P178=7.5127755364752554e-06, P179=7.275538754079225e-07),Row( P0=-492.19142306055903, P1=19.709837198050522, P2=-21.717565878852934, P3=-5.617147857284524, P4=-24.318504250202206, P5=-7.975022012041105, P6=-12.063860121040827, P7=-4.685144096135838, P8=-5.167562472812289, P9=-5.542306058855752, P10=-2.9657302961807392, P11=-1.8422112870737761, P12=5.7058250773295525, P13=3.1777296147028213, P14=6.382723905231922, P15=6.62119656664351, P16=5.873717456275921, P17=7.756341722294327, P18=-0.3887171526927602, P19=5.9705564609302355, P20=2.4935152437161276, P21=5.541354368154224, P22=2.5375532568973753, P23=1.311337010057215, P24=1.3546209885500768, P25=2.5827340711197895, P26=4.568219586241195, P27=6.452110089605464, P28=2.4897159254592838, P29=4.87907507445694, P30=-0.6071531687210736, P31=2.4821418716465184, P32=0.15679333777366336, P33=1.7285914864827792, P34=-0.1912661174837503, P35=0.4867896746908516, P36=0.47426044108861964, P37=0.40118360591867197, P38=0.636229251126014, P39=1.4081602903866162, P40=0.6331048816327113, P41=0.6266905565218085, P42=0.6377880929190216, P43=0.6602247555114534, P44=0.6610517374486343, P45=0.6233664865593395, P46=0.6606841910926751, P47=0.6449607936735835, P48=0.631657905650492, P49=0.644166651049046, P50=0.6214833771200236, P51=0.6244519774430618, P52=0.006803324600893865, P53=0.0005074507153354531, P54=5.036324549360648e-05, P55=7.09495506585582e-05, P56=0.0001953820557674018, P57=0.0004627821128743546, P58=0.0005275251663968111, P59=0.0013719337793404393, P60=0.003134941671538639, P61=0.006786558020911041, P62=0.014776544335640752, P63=0.03557528688391747, P64=0.03533744067119069, P65=0.036646121670816954, P66=0.0483332459798435, P67=0.06447910312322491, P68=0.0966935606076052, P69=0.14745932646696527, P70=0.15019371407072407, P71=0.08793077441963634, P72=0.04107043727479402, P73=0.0331265867855001, P74=0.02798067069160333, P75=0.0329023605386725, P76=0.015856979276334626, P77=0.0239532132434787, P78=0.09230657625870937, P79=0.11891295300543338, P80=0.04614182759325665, P81=0.023002872189244006, P82=0.03523817882788023, P83=0.04275458353297896, P84=0.05532136027185269, P85=0.08559692332393022, P86=0.14055013239169314, P87=0.1912769342509731, P88=0.45544523224312483, P89=1.0933114459452549, P90=1.0973020816776144, P91=0.10367817719027539, P92=0.03326527995363663, P93=0.029255706228491444, P94=0.04515660938483051, P95=0.0708423434948242, P96=0.04897533647221511, P97=0.0507622917358296, P98=0.04365113561849001, P99=0.04881173790456181, P100=0.1748793045271531, P101=0.10983162009335402, P102=0.017655065002850346, P103=0.010097232119331872, P104=0.02390102620460861, P105=0.01034488344650194, P106=0.011474388061930308, P107=0.04297801495185953, P108=0.022153653924001714, P109=0.009172920011764474, P110=0.007449774395683196, P111=0.030684409103983398, P112=0.03918142393232466, P113=0.05483836420422854, P114=0.02817355917102923, P115=0.01650942362043241, P116=0.018866307128597357, P117=0.02501826960962933, P118=0.008797183618610722, P119=0.005567649728802956, P120=0.009751661562796502, P121=0.0071916104675998176, P122=0.004876394171860458, P123=0.005960001103611664, P124=0.004385173200549239, P125=0.004432579931812529, P126=0.00677059554948208, P127=0.004017491814338842, P128=0.003952512098047866, P129=0.0026300619206392007, P130=0.008793807911587844, P131=0.006089762725559313, P132=0.003881213078662711, P133=0.005600679919504126, P134=0.0038799329739657115, P135=0.0014889044427026732, P136=0.002584571558358549, P137=0.008513848487225594, P138=0.009884916277093415, P139=0.009361950109632509, P140=0.008988381702460212, P141=0.004138744186704731, P142=0.006423412110647757, P143=0.0035491361004660977, P144=0.004618817303163089, P145=0.003309641071331044, P146=0.002794041316551016, P147=0.0028343531147990396, P148=0.0016525231461670857, P149=0.0017646936619612284, P150=0.0013521034605316, P151=0.0014638623280515959, P152=0.0009130658103192517, P153=0.0007485891710205426, P154=0.0008593971235504423, P155=0.0015930461180230031, P156=0.002217982629869306, P157=0.003051922989836753, P158=0.003053665667783407, P159=0.0023677506701389713, P160=0.0015987214657839908, P161=0.00099276002775373, P162=0.0006220222824481433, P163=0.0005175606365244716, P164=0.0005922097815170714, P165=0.0007695202017996639, P166=0.0005800345738895964, P167=0.00031559221910815416, P168=0.0002186154453035111, P169=0.00019472661769807668, P170=0.00021293061363979366, P171=0.00021979111296458736, P172=0.00026754135569497016, P173=0.00035230372710929524, P174=0.0002670122372530509, P175=0.00028559317139780253, P176=0.000283420849935318, P177=0.00029773115000585314, P178=0.00019583091331237416, P179=2.623777178845213e-05),Row( P0=-620.726695756111, P1=67.29328875633566, P2=0.13109614006963569, P3=1.5481744456001725, P4=-12.700135579294994, P5=-2.6815643472402897, P6=-9.462025737443835, P7=5.885241129601838, P8=2.160030997696377, P9=-3.6383241409673595, P10=-1.8232785665774713, P11=2.476708986692041, P12=-0.6558974395514597, P13=-3.919267879475058, P14=2.967144797898392, P15=-2.6407346396412987, P16=6.263813455772902, P17=4.512571203118014, P18=4.811964471998286, P19=8.673895378243726, P20=2.27417142337733, P21=2.574545058183474, P22=1.0596922866091092, P23=5.481082713599662, P24=0.5749591590830733, P25=4.400216918136145, P26=3.5808535689626955, P27=4.685708719530321, P28=1.392792703627515, P29=2.530789447289005, P30=-0.38560420040002424, P31=3.3872137026161724, P32=0.8114390264318585, P33=3.0373514352915127, P34=-0.34024809380663495, P35=2.834100472530908, P36=2.159089829768814, P37=2.708066313650278, P38=1.5320070430082429, P39=-0.15119109846362216, P40=0.6565444199041572, P41=0.6415142311748343, P42=0.6705340346252633, P43=0.6794306511835376, P44=0.6867767674180747, P45=0.6882877635681134, P46=0.6663751225030968, P47=0.6663228961683446, P48=0.6945617096859016, P49=0.7776615992638507, P50=0.7995375914970659, P51=0.7263677890467989, P52=0.00030219050290464976, P53=0.00020628864366441251, P54=0.0001410329593094236, P55=4.1827582750124995e-05, P56=4.706417014190669e-05, P57=0.00019343774981527177, P58=0.0005018059299892421, P59=0.0011006282817898213, P60=0.003752342892826667, P61=0.007434352589443037, P62=0.0068773661909205565, P63=0.003960346820583322, P64=0.0034980470991824086, P65=0.0096576779583364, P66=0.012831626592308317, P67=0.005630543837005755, P68=0.003408642862915204, P69=0.008126048467338363, P70=0.017471115066770584, P71=0.014644913883408698, P72=0.006248414765686621, P73=0.007856589579408121, P74=0.006657445264149992, P75=0.006329377377229887, P76=0.006139711408927722, P77=0.0157125160927847, P78=0.027746561138260434, P79=0.04568981323735312, P80=0.07110044806676911, P81=0.049274796923093514, P82=0.01791689172715882, P83=0.00713849410536267, P84=0.005226863776796304, P85=0.0032059088747687504, P86=0.0016249799917298958, P87=0.0017355202699626464, P88=0.003713260320543981, P89=0.003716386227573866, P90=0.0040465025214939205, P91=0.004968662654863153, P92=0.03370137263418095, P93=0.037338918594842216, P94=0.004538938189767484, P95=0.005188015449527043, P96=0.0025809584386177445, P97=0.0014751853752320802, P98=0.0014933242060357153, P99=0.001456461974695128, P100=0.0012799096302550017, P101=0.002753941957477031, P102=0.0019219699661333683, P103=0.0002398474023815921, P104=0.00040132678323719556, P105=0.0003664622898315872, P106=0.00036968710358603314, P107=0.0006918871656499443, P108=0.006157459369985037, P109=0.0068068042046718815, P110=0.0002499480345453735, P111=0.00019013272883025217, P112=0.00013037564517471626, P113=0.00018567728135736647, P114=0.0017662832163340633, P115=0.0010521927492900796, P116=0.0002966946697173598, P117=0.00030617334382845586, P118=0.0003906348897308338, P119=0.0004912503919769265, P120=0.0003983624147521092, P121=0.00037680081698026425, P122=0.00041251162803329785, P123=0.0003447191761075849, P124=0.00016889129114529654, P125=0.00019636089804793842, P126=0.0001928908973124907, P127=0.0002521333284747599, P128=0.00020498586681778766, P129=0.00014346267125568143, P130=0.0001735808847332246, P131=9.302606674689259e-05, P132=0.0001221629052121796, P133=9.106217133722235e-05, P134=2.740996563120993e-05, P135=8.828626738230049e-05, P136=3.49622783170138e-05, P137=0.0002811613328211147, P138=0.00026506199085632725, P139=4.2122825912635186e-05, P140=0.0001800628090012121, P141=6.554146988004151e-05, P142=0.00011719629017219623, P143=5.1850259624388855e-05, P144=0.00017573840434946606, P145=0.00015251648210604893, P146=0.00034347039368492696, P147=9.689774590125832e-05, P148=0.00011039085102972729, P149=0.00014010865566607344, P150=0.00013688341273113686, P151=0.0002111460725510887, P152=0.000272024249247451, P153=0.00044205903122369596, P154=0.00031124499735292336, P155=0.0002663672301840097, P156=0.00025262584446124743, P157=0.00020731048396297638, P158=0.0002445754418189039, P159=0.00046489506253801226, P160=0.0005508162718981254, P161=0.0005656389369416838, P162=0.0011052772349203432, P163=0.0011559513271984512, P164=0.0009952881632967815, P165=0.0006188833041979966, P166=0.0005559615257302584, P167=0.00046293610182867836, P168=0.0002905532731504478, P169=0.0001343100178910857, P170=6.074738915215745e-05, P171=5.3722804188770214e-05, P172=5.9432930673655644e-05, P173=6.91185341276856e-05, P174=4.01093640495222e-05, P175=2.9528618202811993e-05, P176=2.1813089526743555e-05, P177=2.0557882982631137e-05, P178=1.1580045329561996e-05, P179=1.7479266131912428e-06),Row( P0=-614.7230543736787, P1=59.563507886294204, P2=5.773294885256348, P3=4.936584440831364, P4=-11.12894558053838, P5=-2.56903543053331, P6=-9.321023246939703, P7=-1.5172665009013986, P8=-2.200803661100147, P9=-0.3743572550450413, P10=-5.63761628319718, P11=-1.6506529832497674, P12=-0.06785871746785968, P13=-6.7217445810679015, P14=1.5363637146509332, P15=1.195797550298741, P16=3.376634364595212, P17=6.378494523957176, P18=7.758856850650887, P19=6.905375063837668, P20=1.3685015757344752, P21=4.080108230486845, P22=5.717010958639354, P23=5.203547589190942, P24=5.78476546893365, P25=6.585561507000538, P26=2.5643831731519398, P27=2.258014111277837, P28=1.740334786865534, P29=1.8338162964972802, P30=0.14009806673380448, P31=2.98913612076669, P32=-0.9215788716171536, P33=5.148568375598843, P34=0.830755632264697, P35=1.6037324874320893, P36=-1.3831884663446585, P37=-0.13412492577615573, P38=1.2615015577585866, P39=1.8427747984776257, P40=0.6580769805251877, P41=0.5786606623121555, P42=0.5764946804257246, P43=0.6281881531203324, P44=0.6504498535264501, P45=0.659694108634294, P46=0.662701909884441, P47=0.6538647988866363, P48=0.6470312607082597, P49=0.6611042048563631, P50=0.6707803080629505, P51=0.6848486198891741, P52=0.005339652313632079, P53=0.0004591069443004924, P54=9.87174093682603e-05, P55=5.19262897863638e-05, P56=0.00013192047489705505, P57=0.0006731570123297047, P58=0.0013989712605587685, P59=0.002530325016460234, P60=0.009898284876185317, P61=0.033015328041325016, P62=0.0753442870440805, P63=0.07805580430477024, P64=0.04006769399236615, P65=0.03831777876083826, P66=0.09189464509062613, P67=0.16548286771402992, P68=0.0796990243361143, P69=0.020029888787879688, P70=0.03682530520669645, P71=0.07172199108289258, P72=0.09182753456859195, P73=0.10513033767923725, P74=0.11499164203590274, P75=0.049823083529121175, P76=0.022967378867323383, P77=0.019842422952641853, P78=0.03515555415500206, P79=0.06399577779804057, P80=0.06456690884960331, P81=0.06119798577133776, P82=0.0765954343743903, P83=0.06721594410932985, P84=0.07108956247161329, P85=0.03572942943969048, P86=0.009205290695955342, P87=0.0030373937370029745, P88=0.0031119976012584168, P89=0.005470619069394962, P90=0.014507776235748181, P91=0.030318732802617952, P92=0.05483231906476478, P93=0.04144058727132585, P94=0.04626845429070142, P95=0.03975505582961001, P96=0.018509062709231243, P97=0.0027440993589192375, P98=0.007143655074594117, P99=0.028852357791631894, P100=0.012354071277398402, P101=0.0014738414613771236, P102=0.0018141618300581712, P103=0.002718677728375368, P104=0.007382015811769387, P105=0.008641308850766225, P106=0.0025470881048606756, P107=0.001324132683116822, P108=0.0007469019944200529, P109=0.0007798299777780967, P110=0.002265988718929669, P111=0.00867832042313268, P112=0.004002556104520693, P113=0.0010238309693915842, P114=0.003709812288446127, P115=0.0006311276856931679, P116=0.00015053116404739784, P117=0.0018597299456441895, P118=0.0015480430937558414, P119=0.00022583818924915496, P120=0.0015828082435108025, P121=0.0012043716486850825, P122=0.0003535555769217296, P123=0.002028702561889117, P124=0.0007579613125700429, P125=0.0003752568776091805, P126=0.0004814686124883669, P127=0.00047252603156424825, P128=0.0003469203402208546, P129=0.0002130992465237679, P130=0.0004805222616865455, P131=0.0005025626445882257, P132=0.0002626265755280262, P133=0.00022002409133475023, P134=0.0003134055879134913, P135=0.0002519976933913473, P136=0.00021986963724323003, P137=0.0001218131978802231, P138=0.0001248769870106153, P139=0.00015301225513953342, P140=0.00018208613346404223, P141=0.00014134694630365624, P142=0.00017382419397597934, P143=0.00017013797800288858, P144=0.00018852108190349658, P145=0.00026252922094346714, P146=0.0003134087257318441, P147=0.0003415523070125096, P148=0.00042700958309005395, P149=0.0005552064312378735, P150=0.0005082286671813368, P151=0.0005253692567937223, P152=0.0007049680731219108, P153=0.0006981334678317167, P154=0.0005798790178448662, P155=0.0007783805174966619, P156=0.0009478779152676206, P157=0.0013340008556208466, P158=0.0012701477513136197, P159=0.0017822275704149069, P160=0.0014011407710910393, P161=0.0010790180389668607, P162=0.0007355870094811666, P163=0.0003077471160336583, P164=0.0002509367808863958, P165=0.00020620614778210124, P166=0.00028234773693536604, P167=0.00019625468855040057, P168=0.0001776317715361389, P169=0.00010625460948239723, P170=5.742477078202018e-05, P171=5.40274513134559e-05, P172=6.177808558825245e-05, P173=6.196338297430203e-05, P174=5.775361300366811e-05, P175=9.773630302758885e-05, P176=0.00012668665789599868, P177=0.0001413589503078279, P178=6.0443839095839093e-05, P179=7.320454338026695e-06),Row( P0=-634.4889316517912, P1=52.74078249917871, P2=-10.56725220746057, P3=5.77893119012279, P4=-14.48045124547338, P5=-9.797277164757597, P6=-6.8257159179597435, P7=1.043549474429505, P8=-0.02919882371298783, P9=-2.8790093584821195, P10=-2.6830817024783142, P11=-1.100536787101623, P12=-1.6311239780890652, P13=-8.872664111591897, P14=1.6933177767989165, P15=-2.1212040196622537, P16=0.9634238594993494, P17=6.266586717389332, P18=1.4314933613646033, P19=5.4556112652878435, P20=0.655276336378391, P21=4.468781877969508, P22=8.823083723495534, P23=8.936981429306126, P24=8.186918292140305, P25=10.435911395338593, P26=9.181449130704815, P27=7.783714015730032, P28=4.8694388712404955, P29=5.377745844347759, P30=0.5311186655209951, P31=2.748505189710029, P32=-0.5890611317592017, P33=2.981292792509964, P34=-1.625853418097871, P35=0.41738067006151436, P36=-0.9560400000536244, P37=-2.724218443515181, P38=0.8835350910949182, P39=-1.1063859030878906, P40=0.6171101650452182, P41=0.5843038328464825, P42=0.6231185610054311, P43=0.6324415897003702, P44=0.6496953967986837, P45=0.6139040299188581, P46=0.6164350647652159, P47=0.6233341561374743, P48=0.6266344234611767, P49=0.6795830723026257, P50=0.6856623472749644, P51=0.660230223333128, P52=0.00021965938481101184, P53=6.565684078187394e-05, P54=2.6814548740301853e-05, P55=6.174883502732767e-06, P56=4.315248105319683e-06, P57=2.992953822864481e-06, P58=1.2228608032377274e-05, P59=0.0005821927441461804, P60=0.007756019943738822, P61=0.023536623586570223, P62=0.03251211482244575, P63=0.025474669184556997, P64=0.013172737614556108, P65=0.01425295524242327, P66=0.02103193146139051, P67=0.0243427783366131, P68=0.00848008181003415, P69=0.004121459629830467, P70=0.012473606960692463, P71=0.015614941415823725, P72=0.03109525123234868, P73=0.032674719536990665, P74=0.02282077566615645, P75=0.033665563451548454, P76=0.01739721225761253, P77=0.015498372051420444, P78=0.031828088505837224, P79=0.03968891215549516, P80=0.024430313473236098, P81=0.03263914816670122, P82=0.05836247841578559, P83=0.07576862194144032, P84=0.04190365333236874, P85=0.009870989148824571, P86=0.005663513501655589, P87=0.00678927588725263, P88=0.011455460203748989, P89=0.006174526840375272, P90=0.004997978681586033, P91=0.008235697006635245, P92=0.040445258812282794, P93=0.020664752887619615, P94=0.01717543730309982, P95=0.018075245505007036, P96=0.02329952608508192, P97=0.033959906760780756, P98=0.018331926017829063, P99=0.013121599977660275, P100=0.004989477110248821, P101=0.001729874794293071, P102=0.0012679428812746515, P103=0.0018950694377473571, P104=0.002132582632457273, P105=0.0025632069900529157, P106=0.0029555462000204246, P107=0.0019195567516373468, P108=0.002440401250057154, P109=0.003572811418425354, P110=0.0017053396185307385, P111=0.003025178790369669, P112=0.00227111880322069, P113=0.0010354494298312731, P114=0.0019027703324213982, P115=0.0014416315659738206, P116=0.00036160785753911525, P117=0.0029499808119737266, P118=0.001780833338622808, P119=0.0001942031011213829, P120=0.0006862765635458322, P121=0.0010237430506122064, P122=0.00018734444530093473, P123=0.0004582378977842586, P124=0.0004043159752254612, P125=0.0001752781084934421, P126=0.0004050420044659231, P127=0.0005252701249102885, P128=0.00045099840964327143, P129=0.000532365362968475, P130=0.0006011750132803178, P131=0.0005990888992305113, P132=0.00033735844364501954, P133=0.0004693432684718489, P134=0.00047283363292976695, P135=0.00046073378519068174, P136=0.0007672078484376067, P137=0.0007644733229105186, P138=0.00030428232653704893, P139=0.00031262075722446195, P140=0.0002981343977866479, P141=0.00022469559848934604, P142=0.0003222649254876116, P143=0.00042060028280275564, P144=0.0005047678490791966, P145=0.0002942220796798985, P146=0.0003328703979468109, P147=0.00011613908818831428, P148=9.30149672928763e-05, P149=8.025742472306884e-05, P150=6.0948895825700404e-05, P151=7.643401231442271e-05, P152=7.810380611890162e-05, P153=0.00011724669938194341, P154=0.00011618807746254772, P155=0.00011652227683986694, P156=0.00029169429901543215, P157=0.000651782959000307, P158=0.0005416379390672941, P159=0.0005486220663575049, P160=0.000542732138916652, P161=0.0003712258755048884, P162=0.00018480169351477535, P163=0.00021403233519999494, P164=0.00034052165184318483, P165=0.0005523842632233469, P166=0.0004819921508348587, P167=0.00030066520997358966, P168=0.0001802761401879094, P169=0.00010094902776687418, P170=0.00012047800581764718, P171=0.00014471374690412168, P172=0.00010868259453222987, P173=4.6617395883476355e-05, P174=4.739799531664237e-05, P175=3.32100049366391e-05, P176=3.8359233991483704e-05, P177=5.048094813932647e-05, P178=2.0264744051268278e-05, P179=2.3579065159126377e-06),Row( P0=-543.2800573294385, P1=37.145330135565494, P2=-27.229602406203846, P3=-1.843863140886706, P4=-13.241599706180658, P5=-11.228300294446244, P6=-13.193283354430333, P7=5.078088041893835, P8=3.3109022464130002, P9=-4.533105443642861, P10=-2.1237826171480543, P11=4.804967331595216, P12=10.644537628490317, P13=7.084953484923803, P14=21.064303055899444, P15=22.78698852329188, P16=15.036736891184828, P17=7.596834929352059, P18=-1.7895048014890709, P19=-0.9762641589405793, P20=-3.9512161067911444, P21=9.77436822402763, P22=3.856268753837098, P23=-0.6579197363764308, P24=-4.719288815468578, P25=0.9465521952102496, P26=2.6748745355353614, P27=11.22978611875134, P28=4.232558495302874, P29=1.958644313254725, P30=-1.0473056586175868, P31=4.604387915610603, P32=0.29712758108369136, P33=-1.0007141401551114, P34=-5.468630946146497, P35=3.672224607615298, P36=-0.21233461522498223, P37=1.8617590359554008, P38=-3.109603248626092, P39=-0.4117929581230161, P40=0.5876392466668793, P41=0.5856168110514238, P42=0.6339611519058085, P43=0.6582118867490624, P44=0.5492409977122639, P45=0.5127346081736188, P46=0.5478271986658655, P47=0.5284460310303618, P48=0.561817970488272, P49=0.6763160946932602, P50=0.6680021928658948, P51=0.6030686112960997, P52=0.010199668134308502, P53=0.0007643008054723963, P54=0.00012697587599631, P55=6.247964466097304e-05, P56=3.4205296874271316e-05, P57=0.00011909322812526977, P58=0.00041885967324432124, P59=0.0010265975948542594, P60=0.0020408222761002346, P61=0.0036286597160557297, P62=0.004175917340573568, P63=0.0028333194166050065, P64=0.0012259372458097175, P65=0.002218368736982033, P66=0.008478538768300929, P67=0.018840933665474766, P68=0.036715146340197, P69=0.08941932687305622, P70=0.14705683459222865, P71=0.08277040067815664, P72=0.01836262172754274, P73=0.013977014160437265, P74=0.010758390736378753, P75=0.009217846343270386, P76=0.005756263505201731, P77=0.0072522764383690826, P78=0.010891729474710517, P79=0.014885400990129163, P80=0.016271917338893542, P81=0.018381456910574936, P82=0.025197807928627518, P83=0.03422888714907327, P84=0.05886729004630992, P85=0.06465781267162937, P86=0.06149593937310095, P87=0.10664923176731297, P88=0.21102483583775986, P89=0.24382283131329197, P90=0.19106343401709894, P91=0.015326123081027675, P92=0.004979053430735403, P93=0.003176414530089976, P94=0.003583905136769519, P95=0.0037871697942239956, P96=0.0021727597546474923, P97=0.004083886496787044, P98=0.007475112599532133, P99=0.05156837446981858, P100=0.2777909973506899, P101=0.22722764662703124, P102=0.09050796494297213, P103=0.017992348195340808, P104=0.00034997923239944836, P105=0.0006811775981449018, P106=0.002425544150825777, P107=0.006836215304461985, P108=0.01257744914042906, P109=0.03457931372628369, P110=0.06055319811045966, P111=0.19625283194762316, P112=0.016030587510314824, P113=0.00486019842706143, P114=0.013436617050133824, P115=0.08630935320893915, P116=0.07036139482027724, P117=0.04590972738866465, P118=0.02541348086173933, P119=0.011525208477929295, P120=0.008311703492724948, P121=0.014173511968712174, P122=0.008050620343759115, P123=0.01282434874948508, P124=0.01157459823746791, P125=0.010670830802909662, P126=0.008566495925416999, P127=0.008489430602350061, P128=0.0018899858379099743, P129=0.0005752971523896703, P130=0.0007418444023719277, P131=0.0008541670000867901, P132=0.0009130384207678168, P133=0.0013667346153446482, P134=0.001627196558937223, P135=0.0022002167609316762, P136=0.002367784902094855, P137=0.0023122152058216343, P138=0.001893328528744732, P139=0.0006660403266519435, P140=0.0005557996881542258, P141=0.0002384182186565714, P142=0.00030446341404342985, P143=0.00019792039358501097, P144=0.0005673232797998409, P145=0.001212903972215055, P146=0.0014916736610562098, P147=0.0025532735098648795, P148=0.0005704565472584765, P149=0.00031616797826848484, P150=0.00016801587310303026, P151=0.0001641453138585285, P152=0.00017501679835493528, P153=0.0001769806467097206, P154=0.0002781702245593712, P155=0.00018497157722171853, P156=0.00015247347985660156, P157=0.00015115611890081015, P158=0.0001804645611994175, P159=0.00018884952379829613, P160=0.0002879347836706213, P161=0.0001423054590143308, P162=0.00022164637022182868, P163=0.00031650602676015836, P164=0.00011915167489968887, P165=6.789872832322579e-05, P166=4.312946379948522e-05, P167=5.377902005170304e-05, P168=6.21517701608862e-05, P169=3.70131377287659e-05, P170=3.79271552229357e-05, P171=4.6576941688014604e-05, P172=5.8805612049897834e-05, P173=4.106743995217067e-05, P174=4.4313539092110695e-05, P175=0.0001174918539279388, P176=0.00021698466409394372, P177=8.590799708933346e-05, P178=1.4862740985112986e-05, P179=1.358842282903456e-06),Row( P0=-559.8565453038763, P1=16.651675538163044, P2=-29.712840318374404, P3=-11.178768301038486, P4=-18.174855547127358, P5=-8.371133891043456, P6=-7.492389785198805, P7=-1.2137261771792673, P8=3.6495112847395177, P9=3.6589597290951246, P10=10.654751076676632, P11=13.820777414419167, P12=6.3408330384411045, P13=6.038761529041045, P14=-2.8164687663945043, P15=-2.2329887558005654, P16=-3.5439917645461425, P17=-5.93685718694809, P18=1.2292422772122187, P19=0.0332724003736717, P20=1.7258939577076005, P21=1.2461651430036051, P22=-0.9465860062080242, P23=-0.48395178056721283, P24=-1.171082811790383, P25=-1.850484606189884, P26=-0.9808133946102, P27=-0.3279857292515191, P28=-1.2411791192350456, P29=1.6370753107738307, P30=-0.9994327489084575, P31=-0.7920076172164117, P32=-0.8666454279056368, P33=1.1181380192174137, P34=-1.3339605161576804, P35=-0.9657096841584129, P36=-2.24343564731964, P37=1.2975439794794985, P38=0.13265987517053987, P39=-0.20022504502661445, P40=0.4967500885389107, P41=0.4921533767377777, P42=0.4982105155476734, P43=0.4784655677850118, P44=0.5045037297982898, P45=0.5514760257200956, P46=0.5235958978547747, P47=0.5540193633049791, P48=0.5674382675376998, P49=0.6092694138624157, P50=0.5690321401729861, P51=0.5136583499517773, P52=0.0030231857396164743, P53=0.003053690120513843, P54=0.0010118371309005723, P55=9.91757592147344e-05, P56=0.00012358153121047626, P57=0.00011241039373566782, P58=7.119855030235839e-05, P59=4.5700211102561273e-05, P60=3.012670021732424e-05, P61=2.2638531489500054e-05, P62=2.2216889084008142e-05, P63=2.5163394355588396e-05, P64=3.103903189123051e-05, P65=6.265840480590834e-05, P66=0.0001903160852613393, P67=0.0003890520281087315, P68=0.0019565776750546156, P69=0.00933224450700379, P70=0.017381614956779354, P71=0.02838285852199848, P72=0.060498744713451395, P73=0.08781024637731005, P74=0.16957297150499032, P75=0.2818269653312056, P76=0.37573497077386797, P77=0.5662917966718976, P78=0.4344150341834542, P79=0.3001655411630397, P80=0.250067275606377, P81=0.1257059806458639, P82=0.07134916907797793, P83=0.07393735452332358, P84=0.04793006869131562, P85=0.016068855944182368, P86=0.0046120547753597365, P87=0.005073184396032813, P88=0.005937515983032279, P89=0.008172662643701917, P90=0.022945029622353733, P91=0.06575085461386859, P92=0.18415761570753134, P93=0.14132356280612618, P94=0.08702128758338798, P95=0.07710940607020339, P96=0.06478415849470802, P97=0.06185649666035504, P98=0.05429733280953217, P99=0.08785461133361062, P100=0.1452448683322409, P101=0.139473350937492, P102=0.24048660197187252, P103=0.17450945581359845, P104=0.23160177124427095, P105=0.3026174238036445, P106=0.05913834584000897, P107=0.012237763305487363, P108=0.006990501393884324, P109=0.0031782561223223297, P110=0.0036131434632698943, P111=0.00838787714224708, P112=0.018956446705785227, P113=0.04374135777123989, P114=0.056859449680727404, P115=0.032051093625783074, P116=0.022528775104611612, P117=0.0200445074124385, P118=0.01076080903576642, P119=0.007239906235674873, P120=0.028419065884732777, P121=0.03223923498207331, P122=0.00598691314660808, P123=0.0035147182476821166, P124=0.002970337910149795, P125=0.004238286868714403, P126=0.012732678950920695, P127=0.013556225626257455, P128=0.002967056181455634, P129=0.0022271667800586935, P130=0.014635048087485652, P131=0.01235713481632838, P132=0.0032735715702480943, P133=0.0020179156644748214, P134=0.004272113381275463, P135=0.01094840843002426, P136=0.0024272687571130487, P137=0.000961494597548346, P138=0.0016040086199722184, P139=0.008383396361462457, P140=0.0033403903174268404, P141=0.0010704571421256736, P142=0.0009362598254663845, P143=0.0009636380584007856, P144=0.0007885021185274097, P145=0.0012088962274923495, P146=0.0017689749019890455, P147=0.00254854217730574, P148=0.0028582978625220374, P149=0.005362265956772801, P150=0.006319818933671472, P151=0.003966641731883467, P152=0.003555445455473627, P153=0.0013146396836003235, P154=0.001084519675634542, P155=0.0004988743631463815, P156=0.00030715594590162144, P157=0.0002388740780846663, P158=0.000341248509151199, P159=0.0004035233778434415, P160=0.0004557459799321385, P161=0.0005107145645741214, P162=0.0005220364331243426, P163=0.0004797260771013298, P164=0.0005077154725328663, P165=0.0003733978343174902, P166=0.0003819234747209639, P167=0.0003911458487331015, P168=0.00034734037483176677, P169=0.00028198554187347014, P170=0.0002857881006243396, P171=0.00025141434148596764, P172=0.00018461292919288118, P173=0.0001249696903385868, P174=0.00015193187672110344, P175=0.0001903430671030318, P176=0.0001774072270647039, P177=0.00026334615507618813, P178=0.0001651133007051404, P179=2.844429465653658e-05),Row( P0=-481.0253790873841, P1=-15.231529340262979, P2=-42.95943944501651, P3=-20.795549768051924, P4=-13.456058686413344, P5=-5.248032649977823, P6=9.830233607846862, P7=18.12815953123539, P8=7.846508193373886, P9=-6.152394440565267, P10=-18.1872252148011, P11=-5.520328383348741, P12=1.8449014049371146, P13=15.491069048338256, P14=1.8547073977552315, P15=-6.001348260471498, P16=-7.851736448422199, P17=1.738584233716139, P18=5.3751351666032505, P19=-1.6024806173392474, P20=2.170234609290677, P21=6.44644745040618, P22=-6.766568806782524, P23=-8.299057855648211, P24=3.1092414733194014, P25=5.108255320715995, P26=-3.272076227839454, P27=-2.3554551585178864, P28=1.0829117768821612, P29=-0.693115241249853, P30=-2.051639995718165, P31=2.8720606017026022, P32=0.07538362789946551, P33=-0.7880801305662088, P34=-0.8275545350306309, P35=-2.2545468720457724, P36=-2.733632818581578, P37=2.975642942847301, P38=-0.029549717596284105, P39=-1.7803942215757225, P40=0.46551952324183976, P41=0.4289959743319189, P42=0.44575706852902, P43=0.5387391545735825, P44=0.6037185466037644, P45=0.5232618960791382, P46=0.3608535807364494, P47=0.3024039276850814, P48=0.32266388374918875, P49=0.3558729246613847, P50=0.4112819414842881, P51=0.4528745497122825, P52=2.0851097157280082e-06, P53=6.908424522166244e-06, P54=3.248212931267323e-05, P55=3.21035538771347e-05, P56=2.3695495239247106e-05, P57=2.9952626607463244e-05, P58=2.5299005238430667e-05, P59=2.1755832125957906e-05, P60=2.0227077900677636e-05, P61=3.207884347544416e-05, P62=3.539461750446766e-05, P63=3.134106070108408e-05, P64=3.4220953406692835e-05, P65=4.5102199344250684e-05, P66=7.761274627373068e-05, P67=0.00010498250878956409, P68=8.813923677729234e-05, P69=9.832810459901273e-05, P70=9.257663245593389e-05, P71=6.351078893082698e-05, P72=4.670799873309896e-05, P73=4.74174758506366e-05, P74=9.431033758761252e-05, P75=0.00014665624681401473, P76=0.00032013447337829253, P77=0.0013362633089287423, P78=0.0030862192739842626, P79=0.005294437949507784, P80=0.01508275053370702, P81=0.03532685797457215, P82=0.04833281257691168, P83=0.20584959955007687, P84=0.4837669495001491, P85=0.5452696695497942, P86=0.34101046524749057, P87=0.5508361414773802, P88=1.0299139587397652, P89=1.6896894799435638, P90=4.466262237248284, P91=8.535039934897107, P92=12.219072711754114, P93=10.573370810087933, P94=13.372982108525205, P95=3.6892410453446045, P96=0.11937461289005419, P97=0.004074159708395214, P98=0.0024299321346553287, P99=0.0010019607353557493, P100=0.0013828356965155312, P101=0.0014625480067554924, P102=0.000867204042466574, P103=0.010541010297552089, P104=0.06116380451466278, P105=0.2198913027447495, P106=0.22611722931752123, P107=0.13136437362074802, P108=0.1226718410545691, P109=0.10773933632561344, P110=0.4293131726970992, P111=0.689580167260073, P112=0.591803299420339, P113=0.8816507378753673, P114=2.7953738400226205, P115=3.3833034346237745, P116=0.6639246283659181, P117=0.031288596802941236, P118=0.010100580467214354, P119=0.006925613319157033, P120=0.009836372446043629, P121=0.015479517702788463, P122=0.06987648493783291, P123=0.6371638069387239, P124=0.6898693926377464, P125=0.5169046170037637, P126=0.3718539639428138, P127=0.271650217400329, P128=0.1252964442395891, P129=0.007030044476876096, P130=0.003269486297474622, P131=0.003915915505099751, P132=0.008267434578569158, P133=0.010029946815408372, P134=0.02331609205808707, P135=0.0529694264432166, P136=0.09402251801052179, P137=0.03239945570120399, P138=0.004669390895090827, P139=0.00861288039565801, P140=0.012705229542050372, P141=0.004215639174501349, P142=0.012005172752799557, P143=0.010257463447609576, P144=0.004254833287424104, P145=0.0017860490066355828, P146=0.003686635498616872, P147=0.006457229455414932, P148=0.024164851016508917, P149=0.05061452106414173, P150=0.014748316144187434, P151=0.004482876565786504, P152=0.0040875945503462035, P153=0.005816146958903724, P154=0.004327123008554106, P155=0.0011273310383285286, P156=0.0014027800871997894, P157=0.0015050604360102068, P158=0.0010091689623522556, P159=0.0011547524131107036, P160=0.0010930507569677495, P161=0.0023692348368173637, P162=0.00293250585233398, P163=0.0031125329659190916, P164=0.0032411811392103105, P165=0.00320184143521208, P166=0.003911427495059409, P167=0.003980987956161358, P168=0.003900737973246074, P169=0.0027257808340135042, P170=0.002882745655665528, P171=0.0029820849378421996, P172=0.003350473581492818, P173=0.002619796344532151, P174=0.0022028384082834625, P175=0.0014313039985020578, P176=0.0011979181360268181, P177=0.0010704029282836017, P178=0.00042257524484426315, P179=6.391365682416119e-05),Row( P0=-768.7101478084189, P1=39.453246883436975, P2=-16.17162668234471, P3=-10.648111724238921, P4=-18.993630181411564, P5=-12.162697253161069, P6=-12.009817224891583, P7=-7.311339295389778, P8=-5.982151427420535, P9=-7.746228017430351, P10=-7.1357699792144595, P11=-1.8955291610104836, P12=-2.8585876749129895, P13=2.3988435109781268, P14=0.9293966806597804, P15=6.680769763740698, P16=4.82316941777144, P17=5.303732885019243, P18=9.115968923882248, P19=5.751270162373181, P20=4.241265462879701, P21=-0.5716617781344124, P22=-5.98454650471926, P23=-6.374420448730624, P24=-5.2849925510869555, P25=-0.7325755814557606, P26=-0.6411792509234063, P27=-0.9116258373740802, P28=-4.041878312330264, P29=-3.1657100738439503, P30=-1.0560200258350558, P31=2.2656002428719817, P32=-0.43943506103721997, P33=0.6879958964948694, P34=-0.34545841113192216, P35=1.5380876162235513, P36=1.6585127993230186, P37=3.931054576942714, P38=1.845546595000159, P39=1.7857958716453384, P40=0.4675475566325363, P41=0.4376903758211888, P42=0.4418487190521631, P43=0.4514812864247046, P44=0.4805693213892071, P45=0.4028836539227552, P46=0.3686844651987512, P47=0.3522083940619564, P48=0.4068453394894287, P49=0.5316291089034348, P50=0.5363157328817992, P51=0.5164143155257713, P52=3.003473247203777e-07, P53=6.515660553890192e-07, P54=3.997631042126304e-07, P55=3.062424295997156e-07, P56=9.5525734572035e-07, P57=1.5611405359862646e-06, P58=5.756588440103486e-07, P59=5.528334262195979e-07, P60=1.437864756833244e-06, P61=1.2980723127192373e-06, P62=2.3442743223966414e-06, P63=1.9923230547923446e-05, P64=0.0008440766690330781, P65=0.005532771438272686, P66=0.012136339612025737, P67=0.011644148424456853, P68=0.003310455593969207, P69=0.00352713742199441, P70=0.008599232173332607, P71=0.010934018233528506, P72=0.005974548171055346, P73=0.0009615495519190547, P74=0.00010658640600400765, P75=9.92438426970361e-05, P76=8.645183989980584e-05, P77=0.0001541952951290432, P78=0.0009539659063470038, P79=0.003045599042838755, P80=0.002247850251447471, P81=0.002866786451265302, P82=0.004089835820820909, P83=0.0021727499181943117, P84=0.0012395307027541257, P85=0.000648514667146622, P86=0.0001689057174207719, P87=0.0003891102453778278, P88=0.002532395490261045, P89=0.003458177900940786, P90=0.003957100846518708, P91=0.008682846500663205, P92=0.0027076138530156475, P93=0.00162752633044048, P94=0.0029300618976199483, P95=0.0015414410369351682, P96=0.00018885608049405773, P97=1.6010141661335637e-05, P98=1.1177357293169357e-05, P99=0.00018409171679854649, P100=0.0013224092743343108, P101=0.000524498391032164, P102=0.0007136680865649264, P103=0.0006085627808332735, P104=0.00033380900166849717, P105=7.143819936160406e-05, P106=0.00013764258441228164, P107=0.00041292650023349236, P108=0.00017632855586301389, P109=0.0003618928853004262, P110=0.00040777298354756987, P111=9.83443697500972e-05, P112=0.00015522298501103658, P113=0.00010716092372382318, P114=0.00013988837728228737, P115=0.00024538117105494384, P116=0.00022114593645053011, P117=0.00011785694246305117, P118=8.838173558557904e-05, P119=9.827989743216298e-05, P120=5.137070389009268e-05, P121=7.754361094614047e-05, P122=5.440009920336662e-05, P123=1.772598778201884e-05, P124=8.142918935711534e-05, P125=6.737665807907619e-05, P126=1.0030824761884053e-05, P127=1.257069048212725e-05, P128=1.9731348483931976e-05, P129=1.8693560298381607e-05, P130=3.4403676377296477e-06, P131=5.018497403593511e-06, P132=3.925326333482609e-06, P133=7.430134204406567e-06, P134=8.363267061422684e-06, P135=5.3964099957552235e-06, P136=1.0467230967658754e-05, P137=9.583262846514281e-06, P138=5.917498971605833e-06, P139=6.97807769008621e-06, P140=1.001157635219768e-05, P141=9.583539657980513e-06, P142=5.957674129249257e-06, P143=5.300276471815903e-06, P144=6.279971014171844e-06, P145=7.641878717313463e-06, P146=6.66062311828489e-06, P147=4.5716554773348465e-06, P148=4.065697205408143e-06, P149=4.353089443133126e-06, P150=7.379218765670561e-06, P151=1.0589746325728702e-05, P152=8.423425202555908e-06, P153=5.927885123206856e-06, P154=3.831792578927689e-06, P155=2.5645821129759407e-06, P156=1.8470510027300844e-06, P157=1.4584067971680997e-06, P158=1.794018791537823e-06, P159=1.946085412899401e-06, P160=1.724723475333439e-06, P161=2.3934111228084054e-06, P162=4.3798977058330686e-06, P163=6.890379395308583e-06, P164=1.3601888264237427e-05, P165=1.9419978794436508e-05, P166=2.0929030865357923e-05, P167=1.4674480643736466e-05, P168=1.2310609127312122e-05, P169=2.7519960320753777e-05, P170=1.9820616406537985e-05, P171=1.760092576258035e-05, P172=1.5190685559878369e-05, P173=1.1712432235020947e-05, P174=1.979775858348102e-05, P175=1.6486191176972064e-05, P176=9.459323761989e-06, P177=7.339842370492061e-06, P178=3.3411842301562094e-06, P179=3.7902501623423304e-07),Row( P0=-617.6758009489251, P1=41.06573656910815, P2=-41.27876820595742, P3=-11.49249058440356, P4=-28.079624700284732, P5=-9.966415795325435, P6=-8.778445134381252, P7=-7.975782662182599, P8=-5.636094654953108, P9=-8.163150706064327, P10=-12.814010160731561, P11=-7.7082607559262355, P12=-10.437422592340178, P13=-4.307852543971712, P14=-4.652510197644555, P15=-0.15893112646401897, P16=1.5108219321408876, P17=1.3118930617977398, P18=6.119356631395649, P19=4.866302123625162, P20=3.5780925320147654, P21=2.7490194622230044, P22=-0.1609683138618172, P23=-0.912258857380777, P24=-4.3375115740034085, P25=-5.116730407668975, P26=-3.7393891322765485, P27=-1.3680314341805928, P28=-0.4053158097453917, P29=1.1998497172000158, P30=-1.4101724882239006, P31=0.167961548836198, P32=-1.0378536135352006, P33=1.5138596216749287, P34=-1.186054098686145, P35=-1.472212618290066, P36=-2.4824170520062023, P37=1.663532211786188, P38=1.4100802613659515, P39=2.1629628204823326, P40=0.45378557606001396, P41=0.46330530023596656, P42=0.4661762349074143, P43=0.43030649770622137, P44=0.42771312479629353, P45=0.40437602648773424, P46=0.4030976671884508, P47=0.39642203168223594, P48=0.47804457567380976, P49=0.4960713757128506, P50=0.44433180452251664, P51=0.42453095337137614, P52=2.694034144603376e-07, P53=7.016767155089995e-07, P54=1.01380256541102e-06, P55=2.244372665291411e-06, P56=4.787047753895931e-06, P57=9.51477920622543e-06, P58=1.0036886183285247e-05, P59=1.6545316738136416e-05, P60=4.3700168631522644e-05, P61=0.00011047882937585708, P62=0.0006916125338999578, P63=0.003046078811767937, P64=0.00709827840367789, P65=0.012594897562075172, P66=0.016226223297962514, P67=0.013089478194106155, P68=0.018017434092126017, P69=0.017665820363095594, P70=0.0032542055999776128, P71=0.000300477290593977, P72=0.00019690930201091795, P73=0.0002722942642706625, P74=0.0022026705625837802, P75=0.005244612425043472, P76=0.008147154163709563, P77=0.05174056118744204, P78=0.055672075342157915, P79=0.03225206571856258, P80=0.023500756365942746, P81=0.00714332376324368, P82=0.005667530819160653, P83=0.0057069574669107925, P84=0.015463435130286854, P85=0.04740125095441891, P86=0.073047993683016, P87=0.07624513027836892, P88=0.04246252640492609, P89=0.021076353293577387, P90=0.021873488577054206, P91=0.03142755812888957, P92=0.03501772797794377, P93=0.009644525016073078, P94=0.04645174468276496, P95=0.03333043781035424, P96=0.005903233053566561, P97=0.006875897900556884, P98=0.018216956338325523, P99=0.0063102055179648725, P100=0.002331218911595654, P101=0.001944341969393656, P102=0.025335628247232976, P103=0.15931932418909964, P104=0.07371303564238053, P105=0.017304132118563544, P106=0.0029640338806311956, P107=0.0009975303199856365, P108=0.0012540723863174073, P109=0.005635307358315358, P110=0.019545908130912754, P111=0.009185466645967736, P112=0.0034662086904083166, P113=0.0016398496538011056, P114=0.00252110389385415, P115=0.008352864897701987, P116=0.0031257720611276504, P117=0.00253843040427934, P118=0.00424744177844817, P119=0.003929662895263481, P120=0.0026910654754781433, P121=0.0009168091714275358, P122=0.0029192281844254023, P123=0.0026714127866503423, P124=0.0006700572320282489, P125=0.0008481727113773984, P126=0.0009703292543274722, P127=0.00028447608099599315, P128=0.00018430288985191146, P129=0.00045697620130130483, P130=0.00026471196912409645, P131=0.000418203885526229, P132=0.000551955761528779, P133=0.0002908710851705824, P134=0.0005508761395062923, P135=0.00041484550484506837, P136=0.00048214207762450515, P137=0.000531541766715517, P138=0.0002263132363825915, P139=0.0005256990896686211, P140=0.0005563722659105334, P141=0.0009225645572605801, P142=0.0005600769342747439, P143=0.00031345495692140936, P144=0.00022119421268641915, P145=0.00018771985323119865, P146=0.00019907534339466335, P147=0.0002063434581090758, P148=0.00025855673553509356, P149=0.00028812977251577637, P150=0.0003881164102142927, P151=0.0003411487902401836, P152=0.00010076254722332328, P153=5.383029552912526e-05, P154=4.70458767394852e-05, P155=3.232327473352909e-05, P156=2.4613694825757012e-05, P157=1.867142943007365e-05, P158=1.2326337414445283e-05, P159=2.6800432911151023e-05, P160=2.8208493116166283e-05, P161=5.1337021919850836e-05, P162=4.7652369309874407e-05, P163=4.6149372525326906e-05, P164=3.399446395705585e-05, P165=3.6490859832871994e-05, P166=2.1995414950334498e-05, P167=2.368871088466051e-05, P168=2.887589139577301e-05, P169=2.1001656499817234e-05, P170=1.9141045137415427e-05, P171=1.5166500135657083e-05, P172=1.0767468044378352e-05, P173=8.70067884741136e-06, P174=7.159643507790652e-06, P175=4.45429926901474e-06, P176=3.3894287037919946e-06, P177=3.9020464336462635e-06, P178=2.7245373780976606e-06, P179=1.8359230812744254e-07),Row( P0=-701.437664401202, P1=32.72624237763097, P2=-27.24495950708869, P3=-11.13098378846608, P4=-19.21659535214772, P5=-14.15637151146121, P6=-11.822775292391558, P7=-6.977736828317917, P8=-8.55478354048102, P9=-8.727339897512168, P10=-7.344361783743482, P11=-4.492262753279486, P12=-3.7281945626315482, P13=2.0128131470410917, P14=-0.9635823979830895, P15=3.740165893268614, P16=4.517045768477894, P17=4.200207057765772, P18=10.297027080944385, P19=8.81228198478033, P20=6.854603538394762, P21=1.378718571640347, P22=-4.585605099332837, P23=-5.998318150420875, P24=-6.940364448839134, P25=-1.778386795320168, P26=0.6344432062277076, P27=-1.0630710840040734, P28=-2.8023169778222266, P29=-4.464150456550347, P30=-2.45840217094421, P31=0.4381717622357124, P32=-1.9746243589797916, P33=-0.9349346174144608, P34=-3.1530562229343735, P35=-0.15941274793254726, P36=0.6238478196490803, P37=2.9897408250091897, P38=1.6406349712713495, P39=1.9386712358144496, P40=0.4986314008605222, P41=0.5090972424341128, P42=0.5105339212844384, P43=0.5601458728445793, P44=0.5560245975650772, P45=0.4429054579641069, P46=0.44473997856767244, P47=0.4319404047739679, P48=0.5450262697309521, P49=0.6533637064039288, P50=0.579692388700507, P51=0.5230578133968837, P52=2.1116045485759685e-07, P53=2.388244592346104e-07, P54=5.922542086725084e-07, P55=9.504871144585867e-07, P56=1.4556752118472609e-06, P57=1.1607055165486866e-06, P58=4.6596937480835074e-07, P59=4.557154574431678e-07, P60=6.468920100187513e-07, P61=3.0410298461517983e-06, P62=2.515815660382895e-05, P63=0.0001731982546805504, P64=0.0026433318832846915, P65=0.016623596232991162, P66=0.021735167473516188, P67=0.012361306016301584, P68=0.004656549590935398, P69=0.009214961372998895, P70=0.011573993748964914, P71=0.014804757754686248, P72=0.005093866992252061, P73=0.0007948249884509012, P74=0.00017298877292655293, P75=0.00011348509300293458, P76=0.00012890158176454496, P77=0.0008388078730285248, P78=0.003973081599811874, P79=0.01163477879957379, P80=0.018983608680288134, P81=0.00532865150912155, P82=0.002845675915113488, P83=0.001215012371089479, P84=0.0006422163798436985, P85=0.0007012136673733029, P86=0.00078563187510823, P87=0.0011690443911435804, P88=0.0027014779321088236, P89=0.01251037217029935, P90=0.036421078116914286, P91=0.02161135570889346, P92=0.010309947873442393, P93=0.0037178014674179036, P94=0.018756271421870595, P95=0.02316584339645591, P96=0.0013308570129808043, P97=0.0007229385557488614, P98=0.0008167787477450533, P99=0.00126663343458597, P100=0.0029890611175774247, P101=0.0016617039043504353, P102=0.0009403608362041633, P103=0.00854581156658145, P104=0.00845448979421449, P105=0.00033508442790157553, P106=0.000679164208776159, P107=0.0014371680874321142, P108=0.00047627445787195677, P109=0.0004938670613373446, P110=0.017972976855311573, P111=0.007236523042598564, P112=0.00042534357535005255, P113=0.0002690644503764144, P114=0.0001933793955004074, P115=0.0005369541033122935, P116=0.0020573584054704463, P117=0.0006526362947048608, P118=0.0004799180812752621, P119=0.000563207434754383, P120=0.00033637472279320036, P121=0.0007661567354630842, P122=0.00041466280832150527, P123=0.0003757516132885878, P124=0.00010210188009695189, P125=0.00016897249478472752, P126=0.00012605628040476726, P127=7.60323893607537e-05, P128=0.00012116987730067619, P129=8.632250065250084e-05, P130=2.4970058673816666e-05, P131=3.940785787514018e-05, P132=4.9314198406557306e-05, P133=4.5023572437014706e-05, P134=0.00014236557786875278, P135=9.804428428969434e-05, P136=3.062943705834284e-05, P137=3.1219929160719015e-05, P138=1.702368421732633e-05, P139=4.7854019697339435e-05, P140=5.160976630913776e-05, P141=3.0459935073584654e-05, P142=3.319256778579717e-05, P143=2.8185462270251275e-05, P144=4.518898190093747e-05, P145=2.7308992134088433e-05, P146=1.324720403851778e-05, P147=1.5672741735610813e-05, P148=1.8116855943414234e-05, P149=2.3604447198557265e-05, P150=2.942634797134425e-05, P151=3.624561843642826e-05, P152=2.528792298540827e-05, P153=1.856666112026325e-05, P154=1.450482490706318e-05, P155=1.5804572083566456e-05, P156=1.3405717960051062e-05, P157=1.0549351787941249e-05, P158=1.528718817784679e-05, P159=3.266642982820753e-05, P160=3.200378930642847e-05, P161=4.825338158043268e-05, P162=8.565753930302393e-05, P163=0.00011264733870795819, P164=8.648945267690149e-05, P165=9.063177632647941e-05, P166=8.113323451069694e-05, P167=0.00012003592813629265, P168=0.00017682348158269034, P169=0.0001333156881915173, P170=0.00011402212969562147, P171=5.769353669558847e-05, P172=3.130644946760854e-05, P173=3.7305892310213175e-05, P174=2.7084934609484148e-05, P175=2.4898852306264473e-05, P176=1.4328446851026004e-05, P177=8.870266200232343e-06, P178=4.525190014837936e-06, P179=5.604741853431315e-07),Row( P0=-556.344990193529, P1=14.05346287331571, P2=-34.312554077677554, P3=-14.318616327898399, P4=-25.864234954128282, P5=-12.814731683057255, P6=-8.349327697765853, P7=-2.810563789543988, P8=-8.870979775538443, P9=-13.155175173951326, P10=-9.875580528233856, P11=-2.9672930332427483, P12=-1.8069038637203787, P13=9.306201554650071, P14=11.21419463228316, P15=15.665885928810473, P16=11.474050448223757, P17=3.1780289632273515, P18=-3.255102022680687, P19=-3.7940778338703702, P20=-2.9778707919114815, P21=1.6971717492241811, P22=-1.1550371336601928, P23=-2.246956638536141, P24=-5.802945375634678, P25=-3.36474637213985, P26=-0.16097109612330662, P27=6.557582350232264, P28=2.9375918654539546, P29=3.6527919171269936, P30=0.11609458152478336, P31=3.8808081377571084, P32=1.7872348037763626, P33=2.179237300325158, P34=-2.0243555512287563, P35=-1.8876192570844992, P36=-0.027555275817548203, P37=0.5236716620709508, P38=-0.516391486505596, P39=-0.6075400066249884, P40=0.6160287857338133, P41=0.7056144573588349, P42=0.7033339850292831, P43=0.6316681500321725, P44=0.5330579127268352, P45=0.4691461308437302, P46=0.4406498951501996, P47=0.45000595351386163, P48=0.5028242148587346, P49=0.5231565852689678, P50=0.5438513058439639, P51=0.5557356481314071, P52=0.00011395673237183153, P53=0.00010950448770084827, P54=1.6475697875125067e-05, P55=2.8252458364828637e-06, P56=6.490644375922474e-06, P57=7.657519030546262e-06, P58=7.959350304575349e-06, P59=9.793255039046305e-06, P60=2.5431953926851635e-05, P61=7.12562662462023e-05, P62=9.078464144795735e-05, P63=8.291250015607879e-05, P64=0.00012570389882928779, P65=0.0008545767999349694, P66=0.008626357024371641, P67=0.025894364841779444, P68=0.08203013272217566, P69=0.21564389418766403, P70=0.15127241551115014, P71=0.08444456343252445, P72=0.051027485580417, P73=0.0193112316670661, P74=0.003457681058039497, P75=0.0009711422811185722, P76=0.00036680029483241605, P77=0.0003547399898938663, P78=0.00039856503992223856, P79=0.0005452084043817022, P80=0.0009387475982106583, P81=0.000959743226971661, P82=0.0016358009743516553, P83=0.004515114618864758, P84=0.03694409792713541, P85=0.24210115107201932, P86=0.3249443604384383, P87=0.41238577811872074, P88=0.4136629915377263, P89=0.5168706717622754, P90=0.5140659936762005, P91=0.22686844232448042, P92=0.046146315608635975, P93=0.00371978670550452, P94=0.002486523017099148, P95=0.01012456062223824, P96=0.023088952350620612, P97=0.18741176746286836, P98=0.33802270790810124, P99=0.16461295754204322, P100=0.12456980952017564, P101=0.3790255535040104, P102=0.3999212088523656, P103=0.032389780946042564, P104=0.0007863242508221619, P105=0.003863127435974194, P106=0.008236530997932952, P107=0.0163978762815415, P108=0.008400586366340002, P109=0.019610714570188457, P110=0.053509667488177655, P111=0.04524098682263441, P112=0.006249444495315838, P113=0.007673238676226326, P114=0.02562834813954638, P115=0.031971116755064026, P116=0.011158041004349292, P117=0.016053740590248416, P118=0.027224531142387074, P119=0.024266404539941482, P120=0.011206571846249173, P121=0.005928880443663617, P122=0.005085482788982104, P123=0.011462019658390577, P124=0.012074192606348186, P125=0.008775997441202993, P126=0.003499216702819279, P127=0.0024898989286861145, P128=0.003881903336493402, P129=0.002727502857829894, P130=0.0019951435350381882, P131=0.002031542637873264, P132=0.002378021683537851, P133=0.00264337579554504, P134=0.0018184756941910848, P135=0.003270992797432304, P136=0.004585444261132055, P137=0.0020404065199000508, P138=0.0014819478742918093, P139=0.0015518536789937483, P140=0.0015806614894257186, P141=0.0011296329148753233, P142=0.0024071631648043147, P143=0.0018735338913328109, P144=0.0016025838182248083, P145=0.003005959389287225, P146=0.0010265062125649413, P147=0.0013552708590148788, P148=0.0007575492014001388, P149=0.0024008804945305213, P150=0.0013312927670458962, P151=0.001285305658085702, P152=0.0009374893170453197, P153=0.0008922739248874674, P154=0.0004932293524859121, P155=0.0004889414557759866, P156=0.0008089798217128887, P157=0.0009214539590844356, P158=0.0009236026402932364, P159=0.0008987479348351056, P160=0.0009696803866894324, P161=0.001023770304544201, P162=0.0013074835274303972, P163=0.001967580029898839, P164=0.0016984275439341316, P165=0.0017280440802087682, P166=0.001822616149482684, P167=0.0020583910794648897, P168=0.0022974593637022206, P169=0.002336602010994862, P170=0.0020792899463724633, P171=0.002165038142880669, P172=0.0019056451528407347, P173=0.001724201499606279, P174=0.001428993200613644, P175=0.0011708773151635566, P176=0.0007728881093966421, P177=0.0007078517337443667, P178=0.000334813813446863, P179=3.20114811043653e-05),Row( P0=-467.21889454575677, P1=-5.055568168730269, P2=-44.984487967120145, P3=-17.21344800876345, P4=-24.249590151096502, P5=-10.80812255937438, P6=8.534854512273332, P7=14.586086343871376, P8=12.719254115802428, P9=-3.149340666996471, P10=-9.374156955645145, P11=-3.4385691479367004, P12=-1.723955619030262, P13=10.850433163925812, P14=2.6858008663469124, P15=-1.0165325202953124, P16=-7.6318920829988075, P17=-3.059699998628824, P18=4.0922427418654, P19=-0.3535108228364607, P20=-0.7247537316563473, P21=3.0663513946013765, P22=-2.6331124568553066, P23=-1.5914513148147453, P24=2.542664608271029, P25=1.4176228278831127, P26=-2.399941026827196, P27=-1.6235253990472125, P28=-0.5562085181717403, P29=-0.474556888363498, P30=-1.6803240147623664, P31=3.4948064295827543, P32=0.5968553915933977, P33=1.073181402193117, P34=-1.1399488924389138, P35=-1.7687384146634055, P36=-1.8144409274284308, P37=2.1067184314806466, P38=0.4603972832623858, P39=0.022459031915154783, P40=0.5049018170183504, P41=0.5039476802049462, P42=0.5031953122504448, P43=0.5231973791630103, P44=0.5485188163047375, P45=0.5215553467044138, P46=0.4729851391716458, P47=0.43678848836247985, P48=0.4222600505692188, P49=0.4090271178673814, P50=0.40729523091084874, P51=0.4376422891587563, P52=1.5060566389212452e-05, P53=2.8820777037928466e-05, P54=2.1234592391221184e-05, P55=2.0264534155078507e-05, P56=3.66823814557244e-05, P57=3.6141226635104006e-05, P58=2.192191852278655e-05, P59=2.2274692219648417e-05, P60=4.048351302236705e-05, P61=5.0203955870863365e-05, P62=4.548512588633698e-05, P63=3.646387968313103e-05, P64=3.2054715946499184e-05, P65=7.178711204587745e-05, P66=9.483576197328294e-05, P67=8.716762492263114e-05, P68=9.835519342823979e-05, P69=0.00016805277875627574, P70=0.00025008494152410605, P71=0.0007390326538025222, P72=0.0019797821446241744, P73=0.004015383666851734, P74=0.025918925306682546, P75=0.048210001754221334, P76=0.06149980887813028, P77=0.08102657584300002, P78=0.084659220206299, P79=0.0989975625279151, P80=0.13487548852900788, P81=0.16212777470054482, P82=0.23011179134048, P83=0.33042941377238627, P84=0.5832732183185986, P85=0.6630696851961336, P86=0.666067185967398, P87=0.9441426675720729, P88=1.1312209617452362, P89=1.6497338526562078, P90=3.0305960619100722, P91=4.737063914440788, P92=6.132042345982559, P93=13.712061033333708, P94=5.689820753047019, P95=1.0244092878419968, P96=0.21089979038079756, P97=0.06700336871189423, P98=0.06160521147248837, P99=0.04214925443933987, P100=0.05318147603113166, P101=0.03308516235912837, P102=0.02272082669890963, P103=0.02687412164476919, P104=0.0958795582080596, P105=0.731932799099301, P106=0.472003494975444, P107=0.4378067943746838, P108=0.3449216300668211, P109=0.1849664059324188, P110=0.17494250940273967, P111=0.16897610296974025, P112=0.2067311272348853, P113=0.5256715715594861, P114=0.8038918435682966, P115=0.2485877533797575, P116=0.24026130524795417, P117=0.903642454366616, P118=0.44718320491481134, P119=0.20435721419634337, P120=0.27235745876792855, P121=0.3100852595298881, P122=0.2781758674222281, P123=0.38707334162430024, P124=0.21975930931528156, P125=0.1300662729861763, P126=0.11172253418407317, P127=0.14106178254378107, P128=0.08333595553351224, P129=0.045303268034746276, P130=0.04348330062263479, P131=0.07113440705661632, P132=0.08564791055509634, P133=0.07437725175877169, P134=0.05455724689476781, P135=0.05557903025657856, P136=0.15294697654765763, P137=0.02470825031189946, P138=0.03502868490016658, P139=0.024185481606840566, P140=0.013869290531292988, P141=0.014596399960957514, P142=0.02183163962895047, P143=0.042319893371963714, P144=0.0062356567775441575, P145=0.005923689618152601, P146=0.004195604757622207, P147=0.007823449056124336, P148=0.026630729610870756, P149=0.02772250531672866, P150=0.01551505731767105, P151=0.008701817806669877, P152=0.011047217277572895, P153=0.027169495970465276, P154=0.00446453259762125, P155=0.0008552517936069532, P156=0.000989594336586187, P157=0.003729699405227709, P158=0.0014330198422811858, P159=0.00039263102616196555, P160=0.0003019879982613623, P161=0.0003945267198898762, P162=0.00040817185236376527, P163=0.00045089545243854733, P164=0.00038702185004253547, P165=0.0003885902823107162, P166=0.0007241880186633607, P167=0.0008090007609853987, P168=0.0008183974373067947, P169=0.0007575816164580539, P170=0.000514998637497349, P171=0.000484382111085983, P172=0.0004185154941754572, P173=0.0003881800026896086, P174=0.000474916931805526, P175=0.00045256817927916663, P176=0.0005454244179555134, P177=0.00047115392362861536, P178=0.0002682324533755857, P179=3.798790230947529e-05),Row( P0=-568.2368297899673, P1=21.81316243567787, P2=-33.56789301374135, P3=-3.658861905325761, P4=-23.02986557005813, P5=-9.068803656593989, P6=-1.7563696585103594, P7=-7.478207369419562, P8=-2.6701928905171264, P9=-6.477100128528091, P10=-3.1979173086376043, P11=1.0577504116427823, P12=-1.1031251953621908, P13=11.7051362718006, P14=2.2040885643112134, P15=10.595685897922177, P16=5.603248366494759, P17=2.1344669221865904, P18=8.561002766767032, P19=0.41424146709581, P20=0.44676866913172353, P21=-2.0586651843875727, P22=-5.788105914635819, P23=-1.6246744213197037, P24=-2.7832286406902282, P25=0.013769558507645396, P26=0.0021496902149558627, P27=-2.056640629082294, P28=-3.0204548722952533, P29=-0.14620047277978204, P30=1.3895643542890124, P31=3.3500904480488507, P32=-2.111044280340894, P33=0.7877457977843725, P34=-0.6820098580569436, P35=3.0532868292070514, P36=0.17045818427867435, P37=-2.7819213213787, P38=-2.277972682932113, P39=1.0879945722139845, P40=0.5636486187060009, P41=0.5382936791834113, P42=0.5920808721184028, P43=0.5851913180362308, P44=0.5507996891272208, P45=0.4979819942153715, P46=0.5367716327267777, P47=0.5108930217874563, P48=0.4600461168824018, P49=0.5335350792204624, P50=0.5499943631332963, P51=0.5608261959125372, P52=0.00017788912867604094, P53=0.0002312586790222744, P54=9.542077619830686e-05, P55=4.757435939988222e-05, P56=4.913948823735171e-05, P57=3.5358540611879775e-05, P58=2.8845491716914804e-05, P59=2.388704946119539e-05, P60=2.6269256561239026e-05, P61=3.9330805548208173e-05, P62=3.92957250259005e-05, P63=7.860288998970149e-05, P64=0.0009536278447819463, P65=0.01072443664406789, P66=0.03667770572544187, P67=0.05578115568375843, P68=0.03649243916373286, P69=0.07559398482660157, P70=0.12153050940918796, P71=0.12194173046278499, P72=0.11285021519429018, P73=0.057171916795692966, P74=0.07884928845074651, P75=0.17982913848279566, P76=0.0879147997703059, P77=0.011497294152679492, P78=0.0026146374379450183, P79=0.006142752512080799, P80=0.00762807288849922, P81=0.01031982794435517, P82=0.03970576604537666, P83=0.07532299522812583, P84=0.05136885732498283, P85=0.01786192659161472, P86=0.017102558534596763, P87=0.05781251757008495, P88=0.13886213140552034, P89=0.10616414431171904, P90=0.08904354731469467, P91=0.12205217363047646, P92=0.12485231040945236, P93=0.04587873502814238, P94=0.09187573387196374, P95=0.27450131056867033, P96=0.15711097581522512, P97=0.023729621909557224, P98=0.005950138935735697, P99=0.03282355923790171, P100=0.1590211839964365, P101=0.08463178925974425, P102=0.05673157530304229, P103=0.02379017627327929, P104=0.077480313148573, P105=0.24056403291883185, P106=0.06841625144750342, P107=0.0028041141581113672, P108=0.04272574091915456, P109=0.1102086438282871, P110=0.10957833572233225, P111=0.1019826447399907, P112=0.040466420565178476, P113=0.002891710184943078, P114=0.0020957112534522805, P115=0.008792015959344516, P116=0.02234967258758034, P117=0.0895320762047842, P118=0.039532070380102015, P119=0.0058589933274050725, P120=0.006534151853766077, P121=0.029749150250800004, P122=0.0741062986731725, P123=0.018561798593670183, P124=0.008441320214388695, P125=0.015179522908597939, P126=0.022657146041851022, P127=0.00964457362118006, P128=0.003238052057380814, P129=0.007579549321907592, P130=0.02118430754074086, P131=0.004804835786201602, P132=0.003251052349602488, P133=0.00535593765839756, P134=0.0038049197224372467, P135=0.0050796999358519255, P136=0.008506377970809924, P137=0.0016292054815325785, P138=0.006300808867301694, P139=0.005892689542791711, P140=0.001830647071092283, P141=0.002791228293740211, P142=0.0018260906909053375, P143=0.0017859134208161513, P144=0.0009165925532906203, P145=0.0017063833060389033, P146=0.0012870677052122556, P147=0.0033461355107596395, P148=0.002688998981649378, P149=0.00662659617001719, P150=0.00487932517485909, P151=0.0069866360482008566, P152=0.00380851233798457, P153=0.0022420870293028672, P154=0.0008046143380938168, P155=0.0003240229474911591, P156=0.0002564877952280625, P157=0.00034173122218706086, P158=0.0001819871907871799, P159=0.00017126301837578755, P160=0.00018045495530137582, P161=0.00015920713306758738, P162=0.00010321683589285558, P163=8.474687688989982e-05, P164=0.00011332843145200361, P165=0.00017590335084758052, P166=0.00016734705897916604, P167=0.00019229105219381717, P168=0.00018172277723056666, P169=0.0001112192966339387, P170=0.00011756636873010115, P171=0.00010504329992132011, P172=7.337447803335745e-05, P173=9.13213448267092e-05, P174=9.795714775068526e-05, P175=7.542956883004388e-05, P176=6.194631441048692e-05, P177=5.743477812479815e-05, P178=3.16097829865119e-05, P179=4.452009596280563e-06),Row( P0=-696.9776659704606, P1=34.51555636828304, P2=-25.315592506329654, P3=-8.912159975796767, P4=-19.1450328254322, P5=-14.106798670227809, P6=-11.247334949619887, P7=-8.346686320751518, P8=-8.04545555088781, P9=-6.209192726694733, P10=-7.572176161832622, P11=-4.300407210313708, P12=-5.111867920694883, P13=3.828396482936571, P14=-1.2942744758451512, P15=5.233865842079686, P16=4.344496228851124, P17=2.384559682908059, P18=8.509004771428406, P19=5.034215769488174, P20=4.335374143029223, P21=1.5103299125999128, P22=-3.386052182509521, P23=-2.679407009211119, P24=-2.527206635880464, P25=-1.6825790070048288, P26=-2.779048025218356, P27=-3.1834604084597538, P28=-3.221896931099081, P29=-1.0183564722948084, P30=-2.061989331232384, P31=0.6111008425380664, P32=-2.5591761246305995, P33=-0.3705083806917901, P34=0.12485451826195715, P35=0.954465324708564, P36=-2.039596018212661, P37=-0.07292160786069975, P38=0.7754198302155134, P39=2.154137920061847, P40=0.5524902027981978, P41=0.5127012612577042, P42=0.49573632549580804, P43=0.5206935805060638, P44=0.54622341715152, P45=0.4935088800282406, P46=0.47008345561034903, P47=0.5163232832179275, P48=0.547661315487646, P49=0.5485946393726641, P50=0.5277263284595973, P51=0.55572243599602, P52=8.224867611122586e-07, P53=3.753419107764511e-07, P54=1.2528061062887616e-06, P55=2.6487638883960237e-06, P56=3.505694180617857e-06, P57=2.5822232830577904e-06, P58=2.519610735623599e-06, P59=1.8296193708313889e-06, P60=1.7619017054743014e-06, P61=4.820836929140775e-06, P62=3.155075894262939e-05, P63=0.0005149203935865961, P64=0.0023871434610451894, P65=0.011057609149380903, P66=0.02738268496726197, P67=0.03972903928056063, P68=0.013540513887555129, P69=0.00689904646135802, P70=0.0077320234158924795, P71=0.016789817579811194, P72=0.01997223116637161, P73=0.011512148518444362, P74=0.0031978636479826064, P75=0.0014077994365086532, P76=0.0016119532021918947, P77=0.003226475490463566, P78=0.004999399498887936, P79=0.009419486467702317, P80=0.01347551788592189, P81=0.00925297212614453, P82=0.015496519089506933, P83=0.011486463403670166, P84=0.003819573219041075, P85=0.002013505132634324, P86=0.0007179925793462355, P87=0.001064261944847273, P88=0.003861687355409621, P89=0.009862943378664276, P90=0.01798793053715184, P91=0.02332347496379866, P92=0.026792167036763594, P93=0.0051517379532550886, P94=0.01698534269666848, P95=0.03300369034150526, P96=0.005131700289742455, P97=0.004549388472597086, P98=0.003150592713976841, P99=0.0008308159122284029, P100=0.011822617130596196, P101=0.012835745566921549, P102=0.0009493146658741638, P103=0.025942094131802228, P104=0.03674764383434412, P105=0.005680243851762791, P106=0.0024840639320000924, P107=0.009629235657516293, P108=0.005060182259859354, P109=0.0006174866503314967, P110=0.0137989043645022, P111=0.010548886195784936, P112=0.0019560885516681373, P113=0.0022776178500140414, P114=0.001097007088163354, P115=0.0006559182455255889, P116=0.0016237059101585783, P117=0.002311229674867272, P118=0.004307151607320344, P119=0.0009406720299640271, P120=0.0002638249807471519, P121=0.0004397391295074497, P122=0.0015876377922343023, P123=0.0003028440303688962, P124=0.00020783494427277442, P125=0.001767111799025343, P126=0.0014410893442445831, P127=0.0001386356226536871, P128=0.00014576156510987215, P129=0.0002339623341455648, P130=0.00010929263810521038, P131=5.956532830406663e-05, P132=0.00014878871886541556, P133=0.0001544530436934928, P134=0.0003756140927900353, P135=0.0002442192580506974, P136=0.00010207473412034898, P137=0.00022989965856420142, P138=0.00021258369761175557, P139=0.00018864852706353979, P140=6.746115942646976e-05, P141=6.0854189456434546e-05, P142=7.221088180724674e-05, P143=3.406770287511729e-05, P144=4.01156641856884e-05, P145=4.709855856631865e-05, P146=6.440023155915115e-05, P147=8.905631726748873e-05, P148=0.00013425609066156693, P149=0.00034012801285369205, P150=0.00021060750335546413, P151=9.638108477689473e-05, P152=6.653675300737344e-05, P153=2.764121902915753e-05, P154=1.5914719222803784e-05, P155=9.148788616284848e-06, P156=9.085134138004388e-06, P157=7.092501557302946e-06, P158=7.1030436880368126e-06, P159=7.209600916918694e-06, P160=8.609985191208243e-06, P161=1.0958216741470453e-05, P162=1.9657782010626876e-05, P163=1.956910489395616e-05, P164=4.067130154405675e-05, P165=4.983493880943104e-05, P166=2.4090661427611525e-05, P167=2.5340642298225418e-05, P168=4.7710971074878904e-05, P169=5.8842225783140946e-05, P170=7.778948175668734e-05, P171=8.166370093626146e-05, P172=9.027227290906734e-05, P173=4.67806434020761e-05, P174=4.6559724203148804e-05, P175=2.4002037173242398e-05, P176=1.7112666963572575e-05, P177=1.0718290722676103e-05, P178=5.7900672412556895e-06, P179=7.022389301335677e-07),Row( P0=-678.1353323907283, P1=34.86912043217424, P2=-28.127056722600248, P3=-3.8895410762183213, P4=-18.52919664164629, P5=-13.30950470195076, P6=-8.62736459483148, P7=-7.507897819807933, P8=-6.847988394755049, P9=-7.42437346209751, P10=-6.679477563478437, P11=-2.339565370219816, P12=-5.3132544406566105, P13=4.997381216351066, P14=-0.48688014444861333, P15=5.240644427539679, P16=6.6049265737664635, P17=4.635866375065177, P18=9.73103641844588, P19=5.200426768122319, P20=3.7535137412512327, P21=0.56605450934217, P22=-3.9862830642229876, P23=-0.3761256219956021, P24=-1.9678920828202362, P25=-0.0017965946994076224, P26=-3.1063293996411576, P27=-3.8399185046099493, P28=-2.779330179013437, P29=-0.6999637473578825, P30=-0.9972342985207583, P31=1.1133549290599103, P32=-2.0929586069483745, P33=2.4051996008866015, P34=0.5619085459939558, P35=2.185122012143082, P36=-0.9426071743688348, P37=1.190849988475703, P38=2.0233744589493816, P39=1.9133580632656173, P40=0.5829776508252354, P41=0.530449048075453, P42=0.4971787185038832, P43=0.49863140396280464, P44=0.5133215239680753, P45=0.5058830686443545, P46=0.46276013404884064, P47=0.48620753765608465, P48=0.5141117113965704, P49=0.5169891232147343, P50=0.555126850531686, P51=0.5523146131194714, P52=2.112079924748795e-06, P53=2.1156570028550937e-06, P54=1.177198130452355e-06, P55=1.7390812117629185e-06, P56=2.1400672615491427e-06, P57=4.575153983213669e-06, P58=3.920857719700976e-06, P59=3.0581568969555782e-06, P60=3.6435744102910065e-06, P61=4.000364709967529e-06, P62=5.2932283402062743e-05, P63=0.00046202808250384616, P64=0.0015899237874151765, P65=0.008837800780401183, P66=0.026287742739877665, P67=0.04543902225647858, P68=0.021334490878816066, P69=0.008648214117965323, P70=0.009494575151012662, P71=0.020153606479477216, P72=0.016133164072907245, P73=0.021152048391937434, P74=0.007590644137868633, P75=0.0005958687307601916, P76=0.0007643158622865535, P77=0.0022168579649619797, P78=0.0037124831578159746, P79=0.0057053685835656036, P80=0.008367562368123768, P81=0.0074857552740178095, P82=0.009142862196414844, P83=0.029262495657973474, P84=0.028881965026433247, P85=0.015614115642639826, P86=0.004183518569429621, P87=0.002451890022982919, P88=0.0036076643319185604, P89=0.018569246270363163, P90=0.03815760277502062, P91=0.017998504720113127, P92=0.015718064579112363, P93=0.012744803435449554, P94=0.004153949163099367, P95=0.014881518768771959, P96=0.014586437521408237, P97=0.0034840897262292366, P98=0.004062324788418575, P99=0.0009101661134835999, P100=0.004291578334924451, P101=0.009927151594050661, P102=0.0016739483700878867, P103=0.0016591173225743757, P104=0.009333135705165408, P105=0.006331624333561866, P106=0.0009836321733502503, P107=0.0039900696588047345, P108=0.0038946016815378896, P109=0.00040241362620149724, P110=0.0017677996697672068, P111=0.014831835600893201, P112=0.007920894260107721, P113=0.0020299761361578174, P114=0.001327206183386759, P115=0.0009018184696783363, P116=0.0004367902716216565, P117=0.0028034712080847, P118=0.003844543987329932, P119=0.0005969283936608201, P120=0.00023903347111686976, P121=0.0007011323124973253, P122=0.0014045554243338682, P123=0.0006585424729952115, P124=0.0001780684406923204, P125=0.00025915178123447804, P126=0.0012325599102914412, P127=0.0005386508642917322, P128=0.0004621470946100383, P129=0.0005898359534391145, P130=0.0002268750961617607, P131=0.00027026205184017425, P132=0.00036274039780265926, P133=0.00028440795360803827, P134=9.457320285414478e-05, P135=0.00018578064034387246, P136=0.0002658037589493499, P137=0.00010476839602356752, P138=0.0001262927892845726, P139=0.00047010948300389043, P140=0.0003076486090287614, P141=0.00010505016531148155, P142=0.0002773658828562419, P143=0.00011380724558106523, P144=5.2729834408043185e-05, P145=9.671211674683235e-05, P146=3.986366421511079e-05, P147=6.55047380878637e-05, P148=8.636118704484143e-05, P149=0.00010885293185813464, P150=9.434688121235007e-05, P151=8.009065489133094e-05, P152=7.348460853039062e-05, P153=3.473847079206479e-05, P154=1.0918811640985874e-05, P155=8.499969589434548e-06, P156=5.765206821234076e-06, P157=5.093739227456146e-06, P158=4.450239487403339e-06, P159=6.048108385791594e-06, P160=5.850823294326401e-06, P161=7.020452321120606e-06, P162=1.4899066395002632e-05, P163=1.808106908087135e-05, P164=2.44803840359759e-05, P165=4.10297415768107e-05, P166=3.0023807643419882e-05, P167=2.652555559277997e-05, P168=3.880289101471096e-05, P169=4.248199737469662e-05, P170=9.928477616743014e-05, P171=0.0001331515834454688, P172=0.00014871048362956942, P173=0.00011061623206743157, P174=0.0001078627816556185, P175=3.7402729072981077e-05, P176=2.0238302640594058e-05, P177=1.15184067330859e-05, P178=3.227474925484602e-06, P179=3.392672633184675e-07),Row( P0=-558.9441477901307, P1=14.094765628364872, P2=-25.44657360786341, P3=-15.152858248083955, P4=-22.25868517789345, P5=-11.974991090540845, P6=-9.89827029505515, P7=-4.327613232903072, P8=-8.013156733085514, P9=-4.173566053360978, P10=0.28879459393446105, P11=10.235481250074685, P12=8.985229443637591, P13=16.704402030938233, P14=8.916819494943068, P15=1.772973949665096, P16=-5.1589761016206195, P17=-8.480049698452953, P18=-2.5470416571575307, P19=1.0069455582742994, P20=-0.9522781032820192, P21=-0.041945356238107516, P22=-4.793271011403296, P23=0.6430037146477018, P24=4.325587395792184, P25=2.0779850714908537, P26=0.6858023626255982, P27=-0.06091253995246124, P28=-2.5804235075117217, P29=1.813414871851377, P30=-0.40277360841006216, P31=-0.5622515369368322, P32=-2.8056715809375734, P33=2.600439079305256, P34=-0.721035353875657, P35=-0.8375937153862587, P36=-2.274571346299855, P37=1.9316757533578168, P38=1.6986258223435524, P39=0.2729047102933922, P40=0.6015210069801631, P41=0.6531507935069085, P42=0.634289709099367, P43=0.6001525214125282, P44=0.6523029397725693, P45=0.6732485905784585, P46=0.5849760019196966, P47=0.5216602461531037, P48=0.5133579136772611, P49=0.49670301636179703, P50=0.4925701943772516, P51=0.5304484594320547, P52=6.772667275014e-05, P53=5.974370809979222e-05, P54=3.180995540544372e-05, P55=1.901001361751994e-05, P56=1.8379975762644063e-05, P57=1.2713305129153052e-05, P58=1.4130128769946837e-05, P59=1.1533286859618844e-05, P60=1.1212082474423025e-05, P61=1.4853780657768582e-05, P62=2.5102365368400643e-05, P63=2.6676593148156226e-05, P64=4.4149690959252684e-05, P65=0.00038747554584531407, P66=0.0016323968571163067, P67=0.0039751594900075205, P68=0.02148930780690757, P69=0.05346186568223577, P70=0.05371333430687239, P71=0.17298678331854025, P72=0.44104323786766775, P73=0.36098887391428514, P74=0.27584961874795744, P75=0.20393730146737307, P76=0.21381675967566022, P77=0.46505710399786176, P78=0.14980844882267233, P79=0.017190269601502293, P80=0.003060353575021307, P81=0.0007160940227218216, P82=0.0005566197624859809, P83=0.0005643154764922077, P84=0.0009887654861539506, P85=0.0014939103219038237, P86=0.0020424742363099727, P87=0.0033745332381553634, P88=0.01189801246350209, P89=0.04499327356767808, P90=0.15684862766226157, P91=0.3370197427311117, P92=0.20542850376208402, P93=0.24790202212109516, P94=0.25961220166509924, P95=0.09072865403887427, P96=0.10310244473994705, P97=0.17763664786098288, P98=0.21169966879275284, P99=0.054542352697763766, P100=0.01033811988973353, P101=0.08716142334515443, P102=0.2573157352359258, P103=0.3005830569157985, P104=0.05626641755984469, P105=0.4000844118163266, P106=0.37657883016362975, P107=0.08907554532753251, P108=0.018481618749327515, P109=0.009453311193352356, P110=0.02773954848542072, P111=0.08579153302148428, P112=0.06187986063148208, P113=0.006259058454806273, P114=0.006981643659912606, P115=0.038486991749137736, P116=0.01673940059870746, P117=0.07667291552753466, P118=0.13346572091186382, P119=0.09744640954720844, P120=0.009162444970374965, P121=0.00490259369033722, P122=0.00421014409700735, P123=0.0038140686454661665, P124=0.03548772961982254, P125=0.03287756801370444, P126=0.002672077970461047, P127=0.0019181926915409408, P128=0.0036776984739910366, P129=0.004776530487390021, P130=0.002080058550242752, P131=0.0009787774731371565, P132=0.0019532379719279813, P133=0.008445492047486075, P134=0.00221576694214195, P135=0.002780274664313264, P136=0.005901059714713781, P137=0.003185166636459764, P138=0.0014906696510949798, P139=0.0013716468234698164, P140=0.0012555035648129883, P141=0.0004636616779840583, P142=0.0005892633446182584, P143=0.000648646850262905, P144=0.0009135193637095282, P145=0.0017593090535418148, P146=0.0018910284700375914, P147=0.0008921845023286987, P148=0.0020693285677348067, P149=0.0014307611980703715, P150=0.0013098181259436387, P151=0.0011126732936026901, P152=0.000914537736139457, P153=0.0006210276940369295, P154=0.0009680699342249852, P155=0.000625784662997629, P156=0.0007401408182508287, P157=0.0011097533506701943, P158=0.0011752094641366234, P159=0.0016884227843376356, P160=0.0014206402918131536, P161=0.0018502652382574813, P162=0.001959177577959106, P163=0.0019384758421382826, P164=0.0021086261500140014, P165=0.0018852008981995194, P166=0.0016182309768588497, P167=0.0014184868477121622, P168=0.0015905150923832495, P169=0.0019176292212887958, P170=0.0014897642301192745, P171=0.0014911803495160482, P172=0.001079102843437975, P173=0.0010499051642422577, P174=0.0008421528781962844, P175=0.0006273530171222419, P176=0.0005787632613496501, P177=0.0006841160701023584, P178=0.00025457532686194245, P179=3.2749766650930735e-05),Row( P0=-769.968073515611, P1=36.09945680186868, P2=-17.068709098368426, P3=-9.8119984697718, P4=-20.021022181586915, P5=-16.902826529520688, P6=-14.252353048339058, P7=-6.701050522497147, P8=-6.529715387932987, P9=-8.281996957521267, P10=-4.570314156862211, P11=-0.47740268650620554, P12=-0.42819706112610467, P13=2.9939713697799983, P14=-0.9563500516129125, P15=3.2258124757817797, P16=1.513792495357705, P17=2.763540369599584, P18=8.205788630684392, P19=7.429367282477303, P20=6.151797849625474, P21=2.030431225575836, P22=-5.019841460802342, P23=-6.472303570032915, P24=-4.887074099192957, P25=-0.5746097689122545, P26=-1.0828539402970212, P27=-1.691311873506992, P28=-3.8332063860964607, P29=-3.0260070794793226, P30=-3.4125534712048986, P31=-1.1617738992225983, P32=-1.0896968829215146, P33=1.2585003409902735, P34=-1.8578662647594273, P35=-1.2487251637504133, P36=0.852440593751943, P37=6.415222470904577, P38=4.689130453386639, P39=1.2485454949528432, P40=0.48242033226022085, P41=0.45716653778636607, P42=0.4750506053701997, P43=0.464576101838853, P44=0.5292371579640531, P45=0.5698731307509436, P46=0.4772598861143026, P47=0.43479891607658566, P48=0.4733096722444078, P49=0.6175922155227977, P50=0.6565322329353603, P51=0.5755989823481268, P52=1.3326484180624286e-07, P53=9.761489939580147e-08, P54=3.7874415827331864e-07, P55=3.896652662438997e-07, P56=5.514710130209479e-07, P57=1.950137482381562e-06, P58=1.2543378583255104e-06, P59=4.1320136188942084e-07, P60=7.582688175278628e-07, P61=1.5836905943546541e-06, P62=4.113903276881163e-06, P63=7.440434243991334e-05, P64=0.0013480924391459643, P65=0.00746950373930176, P66=0.010485434859596755, P67=0.004041194206647757, P68=0.001115381155824296, P69=0.0030421697511398434, P70=0.0030802066659843274, P71=0.0029837266790724176, P72=0.012735265470262113, P73=0.015123488739226738, P74=0.004288979467542772, P75=0.0007577468103191434, P76=0.0002529466738841325, P77=0.0008384816760542406, P78=0.0032766989068048636, P79=0.0036369914651839456, P80=0.004503644497389938, P81=0.0036871488548733374, P82=0.0023921684603667014, P83=0.0004671436997064677, P84=0.0001631314973181806, P85=0.00011656497625198278, P86=5.831574577320896e-05, P87=9.39160098059068e-05, P88=0.0001603019458175744, P89=0.00028028392888458274, P90=0.0012987881631708082, P91=0.0028533602105923176, P92=0.0076289175663736865, P93=0.0033607306882820816, P94=0.0005379408542630053, P95=0.00018418536470300926, P96=1.454945449534383e-05, P97=1.148134197472351e-05, P98=0.00010552018213684092, P99=0.0007268141342159622, P100=0.0005605509698690524, P101=0.0004051014356540673, P102=8.693816974076452e-05, P103=4.209745696210341e-05, P104=3.937427817606631e-05, P105=7.493373410510831e-05, P106=0.00026848357716146513, P107=0.00012755453843716017, P108=0.00019832446985883114, P109=2.7335997892137125e-05, P110=8.697692840196935e-06, P111=7.047460618430837e-05, P112=0.0002556241752626896, P113=0.00022373742475872855, P114=0.0002902955032069331, P115=1.5306123567414847e-05, P116=6.535262209978456e-05, P117=8.646563052872539e-05, P118=3.283837311702532e-05, P119=3.0332184269431823e-05, P120=0.0001621853582765299, P121=0.00024184155759458543, P122=1.937807576159789e-05, P123=1.2258267114366033e-05, P124=5.047451210380414e-05, P125=7.551098668081493e-05, P126=0.0002936529970140009, P127=2.989533384131678e-05, P128=8.309525551278997e-06, P129=5.115373964102347e-06, P130=1.1666454965867845e-05, P131=9.393294010257568e-06, P132=3.4579964137670377e-06, P133=1.0587821709422609e-05, P134=8.699341691449296e-06, P135=1.798573145934814e-05, P136=8.064228565090431e-06, P137=8.048658453698461e-06, P138=9.911312576638366e-06, P139=1.694800474812503e-05, P140=9.39335924539511e-06, P141=1.030158839463982e-05, P142=1.1553443790021842e-05, P143=6.023280339037954e-06, P144=9.493064302891378e-06, P145=9.009842183617322e-06, P146=5.0559793242680626e-06, P147=8.25745352249565e-06, P148=8.986641695920335e-06, P149=1.1452589573088061e-05, P150=1.2387926628476385e-05, P151=6.646736600766028e-06, P152=8.56715400395931e-06, P153=5.737890597067481e-06, P154=5.1288512223789e-06, P155=3.806176221242572e-06, P156=3.5561917106151227e-06, P157=3.981331616080678e-06, P158=5.257895153198885e-06, P159=4.227264572062585e-06, P160=4.936822368370797e-06, P161=4.7472661264156785e-06, P162=1.2606161212875212e-05, P163=1.544065340865013e-05, P164=2.0067975120096055e-05, P165=1.7313166210975452e-05, P166=2.2782020427652066e-05, P167=2.5976888900168956e-05, P168=2.600896162889386e-05, P169=2.7686071851120233e-05, P170=2.7486703853356987e-05, P171=3.566561000456649e-05, P172=4.1413938665653565e-05, P173=3.432791748335631e-05, P174=3.0333923932724327e-05, P175=3.937776351473125e-05, P176=2.1554386618421462e-05, P177=1.3652618155808715e-05, P178=8.937356133685983e-06, P179=1.479250730348828e-06),Row( P0=-706.5537864162208, P1=39.330399526377256, P2=-30.033412028459747, P3=-9.261758899563045, P4=-17.300371974069456, P5=-13.14485966714508, P6=-10.080382256158911, P7=-5.057170606486943, P8=-6.862870720114641, P9=-7.733147490963308, P10=-7.352517353939655, P11=-4.9049233139593404, P12=-4.052172255205855, P13=2.8103386356078937, P14=-0.3290155084948057, P15=6.761292792355302, P16=5.5915108525150545, P17=2.8503338862433565, P18=5.1029788431049, P19=-0.8582928229609265, P20=-1.0187375001274332, P21=-2.4041174903415357, P22=-4.641827182942143, P23=-4.164613443398214, P24=-5.64819000207339, P25=-5.484049776279476, P26=-5.559508959992787, P27=-2.8356857931908026, P28=-1.6388237702551491, P29=-1.6840608043727405, P30=-3.1859953865545028, P31=-1.0672263791570924, P32=-2.352121691718156, P33=1.431510017618892, P34=-2.0862203299302013, P35=-2.439561220317406, P36=-3.0004729014910256, P37=0.08012289377610637, P38=2.30113891823078, P39=0.8152636414356562, P40=0.6390534761648785, P41=0.6299820790849787, P42=0.6121229781809074, P43=0.5641763962634365, P44=0.5971615814977653, P45=0.5589539250300908, P46=0.5189694997252478, P47=0.5345457783343274, P48=0.5866628852747434, P49=0.6203211910031184, P50=0.5919282077385636, P51=0.613623028332261, P52=1.0765985482424145e-07, P53=1.1305486879317242e-07, P54=1.2905432017900519e-06, P55=4.432941067188197e-06, P56=6.544007781561776e-06, P57=9.78214301597629e-06, P58=5.224812243574333e-06, P59=2.3438300952337797e-06, P60=1.8774143033886303e-06, P61=3.1258952155747293e-06, P62=1.8783421318090075e-05, P63=0.00024647191107858086, P64=0.0020130124033872626, P65=0.0075505086940742755, P66=0.027016487034925114, P67=0.03257345735947371, P68=0.014471706972689666, P69=0.01677576360623742, P70=0.012148214447181688, P71=0.0054913552410671244, P72=0.010637979133735103, P73=0.006533489925899075, P74=0.0006706834558684224, P75=0.00017315776372029394, P76=0.0003727756786224666, P77=0.0023518592791939344, P78=0.004254214301348502, P79=0.003192071031186573, P80=0.010095286303882064, P81=0.01851151920296142, P82=0.02009996873004907, P83=0.012702508280320863, P84=0.01418308912131472, P85=0.012472199545341586, P86=0.012609164431936229, P87=0.01890922000326921, P88=0.0117690213547156, P89=0.00403858769707206, P90=0.004711604548022257, P91=0.006303145341851338, P92=0.008167636698195188, P93=0.0022944601626713446, P94=0.0030194967069659662, P95=0.005732117224237953, P96=0.007921423950541497, P97=0.015448985541378083, P98=0.026038039892881854, P99=0.017424697671104657, P100=0.012202632046860071, P101=0.002937137506504181, P102=0.0005473736754917319, P103=0.00045266754053872274, P104=0.0015343216428300886, P105=0.015629180340515407, P106=0.01639650326218556, P107=0.010748309651785092, P108=0.00545563011348429, P109=0.001391042327049125, P110=0.0004720292955120418, P111=0.004329129390971977, P112=0.005362639934126081, P113=0.005180929300743537, P114=0.00932712551849815, P115=0.0077375484084878845, P116=0.007749646045775238, P117=0.0028164528251604527, P118=0.0013338087173239974, P119=0.0008448515309752084, P120=0.0027952765322122185, P121=0.001234462394675969, P122=0.0009426589272618058, P123=0.00046179061560898155, P124=0.0005522033095887712, P125=0.0006072571840873413, P126=0.0008220468988404247, P127=0.00017860280412281926, P128=8.683871004573718e-05, P129=0.00010813503150149906, P130=0.0001610830779108282, P131=8.139979817595431e-05, P132=0.00014211984966546292, P133=0.0002167672985652078, P134=0.00031273027388544275, P135=0.0003254128117941625, P136=6.763473725219944e-05, P137=9.753253439559337e-05, P138=0.00011388218633644658, P139=4.850555196533338e-05, P140=2.9928968390637294e-05, P141=3.4879549803821566e-05, P142=1.3879586003696274e-05, P143=1.434540492363817e-05, P144=2.8318716178923775e-05, P145=2.3827817422963618e-05, P146=4.9256384748576224e-05, P147=6.295888404213931e-05, P148=3.9792375055101276e-05, P149=7.518905997334667e-05, P150=6.427295875237831e-05, P151=9.231968904570498e-05, P152=8.701897928479274e-05, P153=6.125669086722912e-05, P154=6.369477194725329e-05, P155=4.9446235600777976e-05, P156=5.05928533451555e-05, P157=6.771689730073029e-05, P158=6.729366541464723e-05, P159=7.683981204966537e-05, P160=9.584664325107216e-05, P161=7.543120032126097e-05, P162=7.37358006300769e-05, P163=6.80980795975437e-05, P164=6.425728224386424e-05, P165=5.101823445439618e-05, P166=4.610770848515135e-05, P167=3.232778327531513e-05, P168=2.2996629290608614e-05, P169=2.1193542442046805e-05, P170=1.9199180769582842e-05, P171=1.5266555952483945e-05, P172=1.488797519276453e-05, P173=1.1308060668436335e-05, P174=9.019128652750489e-06, P175=7.3450054567144175e-06, P176=5.318273421980477e-06, P177=5.13837216307222e-06, P178=2.460046438807814e-06, P179=4.176740263807554e-07),Row( P0=-718.4179559403113, P1=37.09768601183541, P2=-11.966062065776175, P3=-12.224525303623352, P4=-15.685344908964325, P5=-15.612772646752092, P6=-12.268538079474991, P7=-2.939145527202725, P8=-7.111332846163637, P9=-5.363753511768993, P10=-7.859958941294766, P11=-5.822263885132517, P12=-9.730181235593232, P13=-3.3476154562684997, P14=-5.205642245846258, P15=3.1163627796958218, P16=6.670626305095761, P17=10.03095677363755, P18=12.559911837247203, P19=9.685056053668465, P20=3.9549998850644696, P21=-1.6810661956030137, P22=-7.352193196743393, P23=-7.471312097609723, P24=-4.691740358806334, P25=-4.961665175265175, P26=-3.1253827744998315, P27=-3.689761623848729, P28=-6.9514637589423955, P29=-3.6141968348780567, P30=-1.7261344574652195, P31=-0.12257038238470064, P32=-1.6935981368081539, P33=-0.5671362524337331, P34=-2.1460621446478956, P35=1.316317989103124, P36=2.051294015420828, P37=4.725656053495904, P38=1.2313184762559675, P39=0.6941355767179387, P40=0.6088438940755592, P41=0.5932209401754088, P42=0.5145217156761992, P43=0.4851464469957132, P44=0.5056276244162998, P45=0.47948990025912475, P46=0.4891694799854447, P47=0.4669932815121607, P48=0.5414530167321483, P49=0.7197121514650417, P50=0.7317214254983295, P51=0.6481449109608273, P52=2.3132763646589856e-07, P53=8.808366317226177e-07, P54=4.903737308966157e-06, P55=5.4797763846599085e-06, P56=9.06028243716645e-06, P57=1.2974718490168489e-05, P58=5.773639644345063e-06, P59=2.351042458064885e-06, P60=4.136625264336486e-06, P61=6.850229984353427e-06, P62=1.7221215861806643e-05, P63=9.365572225017676e-05, P64=0.002147525825996643, P65=0.02257830495582317, P66=0.040031386627278065, P67=0.025141901213031413, P68=0.007894160492655598, P69=0.001840354025433863, P70=0.00045440866842243893, P71=8.980549110857442e-05, P72=4.9582796748661093e-05, P73=3.747643961193475e-05, P74=3.8417985532148235e-05, P75=4.803286006203212e-05, P76=5.9152568389488266e-05, P77=0.00032583439515828066, P78=0.0028329596820584543, P79=0.008795763822063813, P80=0.014154259875389008, P81=0.011815296766665881, P82=0.011178003133107592, P83=0.005611012977075343, P84=0.01048987306886224, P85=0.016603132712400586, P86=0.004011576438264957, P87=0.0004992155736518939, P88=9.339067060119174e-05, P89=6.283745057348515e-05, P90=0.000541723871536717, P91=0.005232531781811491, P92=0.004505707405664773, P93=0.001795537693435158, P94=0.019593345129871546, P95=0.012150155456323343, P96=0.0008657986639346988, P97=9.566843346797563e-05, P98=4.245957815779068e-05, P99=0.00031845913945584834, P100=0.0012027409557293532, P101=0.0005263619849081912, P102=0.0007752659073966717, P103=0.004318468371720072, P104=0.0010459261261823506, P105=9.684248658751154e-05, P106=0.00032984658806969115, P107=0.0004981970758733709, P108=0.00023200673053395902, P109=0.0005790192236322455, P110=0.0026102905873323094, P111=0.0005047370573565492, P112=0.0003179848160180588, P113=0.0002144802111416, P114=0.0003383481486291851, P115=0.002417786202339339, P116=0.003887777802628749, P117=0.0005277438833144112, P118=0.0002201734822816595, P119=0.00012863660982591143, P120=0.00037595982413731084, P121=0.0009089573460050549, P122=0.0002255235652996076, P123=5.682353917535602e-05, P124=9.281929745286401e-05, P125=0.00010684854245119543, P126=3.120686945906336e-05, P127=4.242167331864462e-05, P128=4.777056898396181e-05, P129=2.2963864560082454e-05, P130=1.091013565088861e-05, P131=1.498194214546388e-05, P132=1.5376201152603356e-05, P133=1.4028471439378768e-05, P134=4.1758780124832235e-05, P135=1.8789739741675815e-05, P136=9.993986178814742e-06, P137=1.749890939508155e-05, P138=2.246165566853196e-05, P139=3.215556863791335e-05, P140=1.9346785389420407e-05, P141=9.432119172522191e-06, P142=4.950087769775361e-06, P143=4.062839688709847e-06, P144=3.896403101866029e-06, P145=8.788773318064872e-06, P146=1.6885952891644352e-05, P147=1.7804057588255126e-05, P148=2.3555810485491997e-05, P149=4.9650883793333894e-05, P150=3.574149073904366e-05, P151=4.655958846338206e-05, P152=2.9827336003192992e-05, P153=5.868871958639232e-05, P154=9.248270657381045e-05, P155=8.743335638421775e-05, P156=6.981309542013314e-05, P157=0.00010280528888829722, P158=0.00012438810152783843, P159=0.0001459119195242605, P160=0.000195147522599613, P161=0.00021067707989736294, P162=0.00041077408409188003, P163=0.0005487550171340671, P164=0.0004913047727822322, P165=0.00043387277898154514, P166=0.00023785654763196638, P167=0.00019339758085441588, P168=0.00016582586258926134, P169=0.00017261175889841566, P170=0.00018797067567253133, P171=0.0001458946211073962, P172=8.96589543383413e-05, P173=8.848785410995081e-05, P174=7.641712276184271e-05, P175=4.557314820236346e-05, P176=2.920685251084998e-05, P177=2.626355469519425e-05, P178=1.7354862132619083e-05, P179=1.6677047815639732e-06),Row( P0=-737.0803635166304, P1=44.00458685430645, P2=-18.08455582226886, P3=-16.232296561092724, P4=-19.881792303694976, P5=-13.554673697318588, P6=-12.440341506176777, P7=-6.386890593557821, P8=-8.848195183645917, P9=-5.220362578802087, P10=-9.386659940520532, P11=-5.269816602326636, P12=-2.967187815114603, P13=0.616376127009801, P14=0.44898003135510367, P15=8.234590660051076, P16=7.2137743423622345, P17=7.97865814999928, P18=10.362760069344112, P19=5.258824401257887, P20=3.542421806460014, P21=1.2276184543824826, P22=-3.948008574100399, P23=-6.169721220806738, P24=-5.213450239850378, P25=-4.604944688212723, P26=-2.616855950266694, P27=0.7264681806077367, P28=-1.9884430425548856, P29=-0.5869843748959735, P30=-0.5646457871664378, P31=0.6425762666278082, P32=0.35231372116564574, P33=1.6301166119441877, P34=-3.0854661837047246, P35=-1.4103193006382333, P36=0.39570706900737257, P37=4.866564348839533, P38=2.6765272455628013, P39=2.0747184289885063, P40=0.5740375855624988, P41=0.5746850048559694, P42=0.5612704338342164, P43=0.5132848951369551, P44=0.5144331952171189, P45=0.5143588528075371, P46=0.4650175572982225, P47=0.4778458860163778, P48=0.5594280144889734, P49=0.6396600892170166, P50=0.7048304432788424, P51=0.6146052217177183, P52=6.662494413733059e-07, P53=1.1920694774448527e-06, P54=5.429254743387108e-07, P55=2.624264295583882e-07, P56=1.3060995638006893e-06, P57=3.266712441427621e-06, P58=1.4875991434500762e-06, P59=5.765721830059818e-07, P60=1.0470981734042212e-06, P61=2.5754780740980932e-06, P62=1.2983294236009847e-05, P63=0.00018458379297173913, P64=0.0015407837398973543, P65=0.017308617626505294, P66=0.019999026972169417, P67=0.007692543168951927, P68=0.0070696123306851575, P69=0.01079186044419111, P70=0.008469103556352262, P71=0.002751254673995715, P72=0.00023270978273831557, P73=7.39028487796514e-05, P74=3.832592032504356e-05, P75=4.593432997951449e-05, P76=0.00012789067790539596, P77=0.0010539402656582766, P78=0.001776409096729084, P79=0.006174241998179111, P80=0.015471568506818896, P81=0.007209512937415265, P82=0.0029035248938578755, P83=0.001934021062352109, P84=0.003833090832457955, P85=0.0027221522496579044, P86=0.0028467636172233067, P87=0.003374913313557256, P88=0.003104650859002563, P89=0.0047742034339743914, P90=0.002518066226149449, P91=0.0019436992290589629, P92=0.003985656332237938, P93=0.0009846462801999197, P94=0.0001489112167302958, P95=0.0006440110962506792, P96=0.003768377622702901, P97=0.004910874636251229, P98=0.005700813812226848, P99=0.0019106015916758375, P100=0.001725143965980111, P101=0.0013212998862159443, P102=0.00022986381543597947, P103=3.690320982208671e-05, P104=0.00041943028593261324, P105=0.006939209259393819, P106=0.0035797637852568328, P107=0.0018029234350272613, P108=0.0005385117518311419, P109=4.9671384946576235e-05, P110=2.7658183410563943e-05, P111=0.0007032989729239524, P112=0.003064536633120893, P113=0.0016939573616325232, P114=0.0008431488667261117, P115=0.00015805521035761166, P116=5.1380480023440375e-05, P117=0.000177451033115106, P118=0.00016116797992477182, P119=7.474907917802813e-05, P120=3.753813974998875e-05, P121=3.678035833530226e-05, P122=5.9174015658130865e-05, P123=5.384613648033668e-05, P124=1.7244867807383492e-05, P125=1.2842313669861335e-05, P126=1.2538523026344356e-05, P127=7.717522395174128e-06, P128=7.037105032288739e-06, P129=7.3849395961138246e-06, P130=4.72453839985104e-06, P131=3.0752124010382797e-06, P132=5.181431441465608e-06, P133=6.110508355331579e-06, P134=7.6263939625319836e-06, P135=1.4088391419624553e-05, P136=1.4649521791699136e-05, P137=1.5138594240657783e-05, P138=1.5253174463173462e-05, P139=1.564688502571882e-05, P140=1.2113506421369636e-05, P141=4.8728665920433904e-06, P142=3.1824152592427514e-06, P143=2.6535985238103396e-06, P144=2.8648151671105617e-06, P145=2.449424393916364e-06, P146=2.840221235862358e-06, P147=5.090932120769226e-06, P148=9.314093533150264e-06, P149=1.1978918436413408e-05, P150=1.4444039537999203e-05, P151=1.612976485880702e-05, P152=1.3080503312167517e-05, P153=1.2120917163155875e-05, P154=2.181576001947069e-05, P155=2.473138137040508e-05, P156=4.0624738598939106e-05, P157=3.473726364831631e-05, P158=3.5065731053249536e-05, P159=5.035767801160787e-05, P160=4.897883572812476e-05, P161=7.336009022298459e-05, P162=9.634429083508861e-05, P163=0.00017439140305964236, P164=0.0002671343101331659, P165=0.00022893996087705931, P166=0.00014569739024020842, P167=8.640424583896544e-05, P168=4.541568458288022e-05, P169=3.636425329105154e-05, P170=3.501847303239824e-05, P171=2.8733797028941518e-05, P172=3.651207033076448e-05, P173=2.824669963608812e-05, P174=2.1659482911237986e-05, P175=2.6088271925432873e-05, P176=2.145004765681916e-05, P177=9.067792614337528e-06, P178=5.788725782246246e-06, P179=4.5213697928037084e-07),Row( P0=-539.1350812232539, P1=15.978907713798947, P2=-39.43095817232446, P3=-12.073795480091144, P4=-22.17754260610804, P5=-7.466104545902991, P6=-7.468442137128184, P7=-1.1365042987131775, P8=-6.672909898866392, P9=-8.900639790272383, P10=-3.156161701168862, P11=2.265231833069781, P12=-2.241049665349858, P13=9.303395841712476, P14=6.352896817535646, P15=5.575065223565764, P16=5.446870825209736, P17=-0.242477892196855, P18=0.8553537927310799, P19=-2.554545405185065, P20=-1.583768492114527, P21=0.48238607030690744, P22=-2.0348488143513377, P23=1.1532177037249831, P24=-4.457168102442671, P25=-3.280846381076149, P26=-0.7728521152481759, P27=2.3763127197213993, P28=2.362727653146789, P29=1.3241691660839816, P30=0.19470534151422852, P31=1.696836124393554, P32=0.7220884166782219, P33=1.8274038062163596, P34=-3.6673039271060794, P35=-0.4134801874628186, P36=0.513529588444576, P37=0.37759354648707644, P38=-2.4310209511741134, P39=-2.508726924342165, P40=0.6318852321122247, P41=0.6711084149245694, P42=0.6528776448846944, P43=0.5491865511586116, P44=0.5179578823193367, P45=0.48360933811381984, P46=0.5246064004940572, P47=0.5073061947065576, P48=0.5597398307097253, P49=0.587138994476434, P50=0.5827122251917755, P51=0.6089760355184876, P52=0.0004926168438419496, P53=0.00046942245077214817, P54=0.0002003860382797127, P55=0.00010126238672285421, P56=9.323970613719211e-05, P57=5.869210722201142e-05, P58=4.136775296536237e-05, P59=3.551713879605161e-05, P60=4.064220127850367e-05, P61=7.136171235385126e-05, P62=0.00012188596827001796, P63=0.0001927456831922594, P64=0.0008504598816864425, P65=0.008361797962744739, P66=0.028534116588653298, P67=0.05187764756404373, P68=0.03728516607749818, P69=0.06516234552591917, P70=0.036636099062369774, P71=0.014664320520351923, P72=0.019353764732811113, P73=0.015449939880085153, P74=0.08877985253232185, P75=0.10330597040180158, P76=0.057857253764773145, P77=0.057880872166742384, P78=0.0444317777803804, P79=0.050505437343992626, P80=0.1536592011025954, P81=0.2734657436113144, P82=0.13879470729701837, P83=0.04222169710134719, P84=0.0428840140943079, P85=0.04592253150995333, P86=0.09787706123502307, P87=0.2513810849207547, P88=0.3100188762781396, P89=0.10769949857680165, P90=0.05032223003972081, P91=0.05576726462303256, P92=0.026842878465389023, P93=0.011907733224660535, P94=0.1752264674856706, P95=0.36276465287626, P96=0.33958993750216904, P97=0.26210597093499, P98=0.32510031061873007, P99=0.1283629541820359, P100=0.13733831079239775, P101=0.10881230790012257, P102=0.13614575066127688, P103=0.0856228186678981, P104=0.05943103199348956, P105=0.019096640065353224, P106=0.014611897434843358, P107=0.04518557635513096, P108=0.1345685823217093, P109=0.12263101093121326, P110=0.028419309909268522, P111=0.012114942610955179, P112=0.007617852804030938, P113=0.007155722549053956, P114=0.020359751711095567, P115=0.04279440195786481, P116=0.012894464567275879, P117=0.010796557068736721, P118=0.07179207159750674, P119=0.10773978308340575, P120=0.010459116374714512, P121=0.003240085191926633, P122=0.002343996106974511, P123=0.0021627991536316404, P124=0.013302482202412112, P125=0.01987146580956889, P126=0.004682927405098565, P127=0.0032519591009639304, P128=0.006287426388153128, P129=0.006034012961825284, P130=0.007409383957598515, P131=0.005418733040120738, P132=0.002722196188431393, P133=0.0064539294852610585, P134=0.00287210164903918, P135=0.0016985216587587391, P136=0.00375894696838342, P137=0.005465852401760716, P138=0.0023824797465535645, P139=0.002598607030637966, P140=0.005351138831600797, P141=0.005289332210027584, P142=0.0024748257713232014, P143=0.002918683518250498, P144=0.002571943024370138, P145=0.0028738144005100784, P146=0.0021405749626489945, P147=0.0030387937280823563, P148=0.0032964007256132584, P149=0.0028671224097364034, P150=0.0039329862767035215, P151=0.003607425141544916, P152=0.0038310014068920406, P153=0.0024896596207919126, P154=0.000864077992561573, P155=0.0005186392461583219, P156=0.0007693160704816511, P157=0.0006126771310159666, P158=0.0011783966195847689, P159=0.0010812706581705205, P160=0.0016518669769568384, P161=0.0010331886694010188, P162=0.0017371254249284574, P163=0.0020066427041265243, P164=0.0021044002366638315, P165=0.0016849347921631635, P166=0.0014006532467412563, P167=0.0012568825697216398, P168=0.0012447306126107758, P169=0.001083422854030699, P170=0.0005213550493700142, P171=0.00046731653059232757, P172=0.00031855385760792165, P173=0.000307158850282749, P174=0.00023513160513172275, P175=0.00019904049266865042, P176=0.00018008563486412548, P177=0.00018121014012543036, P178=8.864138104186971e-05, P179=9.058378338747726e-06),Row( P0=-565.4575122112533, P1=16.917419404515684, P2=-22.38558634986886, P3=-10.74343636449488, P4=-17.17016650221408, P5=-7.043490227277355, P6=-9.670180490339485, P7=-4.149538346724414, P8=-3.9106734457745254, P9=-4.656925006845907, P10=3.485049468068055, P11=12.620280523101618, P12=14.399415483030483, P13=19.731850053203807, P14=10.348570755628057, P15=4.647192719189536, P16=-6.773301459440994, P17=-8.659950866167895, P18=-2.0533832941990413, P19=0.7106015951164475, P20=-2.01238631739058, P21=-1.6501563605709582, P22=-5.618292521385066, P23=0.19467446420105042, P24=3.2458601723208136, P25=1.3324322017192722, P26=-0.6156108080506294, P27=-1.652616831286885, P28=-1.6270527170409674, P29=2.2315549403152537, P30=-1.0741292101077797, P31=-0.6149044046542393, P32=-0.5209063637372163, P33=2.076179426926872, P34=-2.2898874232698496, P35=-1.5031308339567646, P36=-2.386574849654436, P37=1.2239720657617035, P38=-1.7710834741468902, P39=-1.8974401536386691, P40=0.6254711011695413, P41=0.647054392262635, P42=0.6379784075123792, P43=0.6185917307740393, P44=0.70056405720895, P45=0.7258989388781792, P46=0.6544955952574609, P47=0.5671670609896822, P48=0.5507204836156929, P49=0.5424234811228051, P50=0.5257075703435925, P51=0.5592816907059139, P52=0.00048608282745024394, P53=0.0005092715903119883, P54=0.00037909915875241674, P55=0.00013653306456643682, P56=9.681316305505497e-05, P57=6.492681718741665e-05, P58=3.474624973841804e-05, P59=2.1140538518238026e-05, P60=1.3547427852662065e-05, P61=1.8065833413325406e-05, P62=1.7166942509938772e-05, P63=8.864869540986224e-06, P64=8.194111530956416e-06, P65=6.388676699975211e-05, P66=0.00046679501994367366, P67=0.0028219159103639087, P68=0.02186062508146799, P69=0.05979287140921924, P70=0.07978583498525599, P71=0.12070764821630219, P72=0.27104223926794974, P73=0.3505359923906271, P74=0.6993744465815533, P75=0.3646671751119042, P76=0.16721567848776767, P77=0.10973409151835474, P78=0.025044294250255218, P79=0.008070376986717075, P80=0.001489082929886456, P81=0.0005266238641481913, P82=0.00033361239917526126, P83=0.00028366246257794097, P84=0.000671346534486946, P85=0.001465643694562458, P86=0.002078091906641799, P87=0.009765766099223841, P88=0.01972344554540417, P89=0.01812278672791986, P90=0.03765028913053853, P91=0.14184895433539885, P92=0.34134139389914187, P93=0.29609780386239215, P94=0.22281711773565174, P95=0.04417205462488144, P96=0.006462390668398271, P97=0.006189548835101741, P98=0.001759770593900893, P99=0.0011447748095449062, P100=0.003397091198031372, P101=0.0023865057331747272, P102=0.0027867098586734584, P103=0.029429992293240302, P104=0.21885320922960552, P105=0.28612183926253076, P106=0.2925386078491527, P107=0.08035180277826393, P108=0.0014573435264235715, P109=0.0021848667210867096, P110=0.0031819719104049047, P111=0.0020470831604674665, P112=0.0024566691848829135, P113=0.0067549068755980955, P114=0.015487315150806983, P115=0.0601085908143155, P116=0.014949432572100421, P117=0.003630229972348027, P118=0.0021848865571531297, P119=0.01153111436728158, P120=0.041192733344152466, P121=0.014189904405671527, P122=0.004985339029161903, P123=0.005166773508586748, P124=0.0021998877351236115, P125=0.0038488520562315106, P126=0.0036729940176453346, P127=0.006134044762842826, P128=0.0017354785146971804, P129=0.0034668368703946986, P130=0.0029929512538663166, P131=0.0020351573717938814, P132=0.0009005140220859229, P133=0.002331288600505978, P134=0.003550070469492014, P135=0.0024010780532200495, P136=0.0020618109771951675, P137=0.0013000587276837626, P138=0.0013871848561369388, P139=0.002972193811537832, P140=0.0022502868099778355, P141=0.0019010580933821867, P142=0.0007854433555499093, P143=0.0006270823274125209, P144=0.00044835088474939633, P145=0.00040237832172239786, P146=0.0012455825237498894, P147=0.0014021654542464084, P148=0.0010809105114895836, P149=0.0023464868801489976, P150=0.0030890869164213602, P151=0.0018032572447774773, P152=0.0038189560824015904, P153=0.0018878143914274705, P154=0.0007368952779554761, P155=0.0012760637771865974, P156=0.0013919032874251075, P157=0.001506359182473199, P158=0.0017712141397897368, P159=0.0014482781962051607, P160=0.0018249852534483304, P161=0.002211965148242633, P162=0.0019054633202238696, P163=0.0021768670709892916, P164=0.0024968954439060042, P165=0.0015045731858132861, P166=0.0011682733853210993, P167=0.001098178841125792, P168=0.0013102058263231775, P169=0.001223680631469307, P170=0.0014803952136494781, P171=0.0010698205953144873, P172=0.0009087632042374294, P173=0.001334676801190035, P174=0.0015821231421190008, P175=0.001348942162543654, P176=0.0007762039231002569, P177=0.0010385649746493166, P178=0.000622781088164573, P179=6.57307560269613e-05),Row( P0=-547.8986613252492, P1=13.955449951113593, P2=-28.111275397410466, P3=-10.898782808392072, P4=-20.946041489323264, P5=-9.716681494793384, P6=-7.14844232395714, P7=-1.9942901877427186, P8=-1.8934809130486014, P9=-5.303745188731916, P10=4.733783391529355, P11=8.454648902191963, P12=5.45564880972535, P13=13.098457746994299, P14=0.7207710279819475, P15=0.11104998069269595, P16=-8.05038240840686, P17=-7.196633946666746, P18=-1.450980771860702, P19=-0.8718567362631038, P20=-1.5015570607027084, P21=1.7752890296523625, P22=0.41083622099742234, P23=-1.0436136945857089, P24=-0.9383420189239333, P25=-2.2750310555699897, P26=-0.9173016983433995, P27=1.651026364048558, P28=-0.800984972065252, P29=1.0845270082318226, P30=-0.7719624089454372, P31=0.5273140456600709, P32=-1.944394974174122, P33=0.767020874494046, P34=-1.8199824265074798, P35=-1.4424174411464679, P36=-2.269397105424787, P37=0.029606386674548197, P38=0.13410811521310062, P39=0.5618963028232776, P40=0.5857737503720154, P41=0.661025456714919, P42=0.6801092677211033, P43=0.6123442224349982, P44=0.6400329419196555, P45=0.6078963725451133, P46=0.5703846766587048, P47=0.573032191497981, P48=0.5711594492990179, P49=0.5865731941130433, P50=0.547280461032974, P51=0.5750142806173623, P52=0.00017691006373744445, P53=9.838966604697015e-05, P54=5.440530545504071e-05, P55=5.7427480267653837e-05, P56=5.2930474001053214e-05, P57=5.750514550055526e-05, P58=4.014504233802827e-05, P59=3.729326265334389e-05, P60=4.8003858923390536e-05, P61=8.121718726028091e-05, P62=0.0002054200175201716, P63=0.00029857461474918083, P64=0.00021164889262892823, P65=0.0001549715156708309, P66=0.0003278289870358048, P67=0.001194967775196687, P68=0.009193808233471342, P69=0.04069473596517318, P70=0.05140234812264409, P71=0.06531086875467457, P72=0.10648555197074588, P73=0.09147159640967542, P74=0.19310026614247383, P75=0.44831961609651716, P76=0.521909316702453, P77=0.49233661168382115, P78=0.6320271931141059, P79=0.5013569240006277, P80=0.053011742060187265, P81=0.0015543856787504137, P82=0.00042667389872857835, P83=0.0004877714141121822, P84=0.0009479904437619468, P85=0.002682275987138585, P86=0.0241734980614969, P87=0.07676590387137848, P88=0.06520465377273296, P89=0.1090628798411393, P90=0.44702902777500386, P91=0.5479875866299854, P92=0.39405038818534727, P93=0.3292213566586617, P94=0.29697736951436887, P95=0.11993498643357793, P96=0.025152960634192617, P97=0.0100321657859451, P98=0.009607953636029315, P99=0.03193630866130821, P100=0.046581467839514897, P101=0.019626301510867604, P102=0.039956870499235506, P103=0.08740656140669223, P104=0.06704663623372455, P105=0.13377652667286866, P106=0.7007943701900597, P107=0.5631245096075221, P108=0.09395087662850106, P109=0.01373904354040522, P110=0.007696267438233385, P111=0.011262491340673055, P112=0.019711774028601282, P113=0.02350870788411577, P114=0.016232566036859533, P115=0.06508910573788307, P116=0.029737949502778572, P117=0.020829790344616423, P118=0.02545735375547518, P119=0.020006724396881656, P120=0.018452934953063892, P121=0.02900885219565752, P122=0.006424658984495813, P123=0.0035983587257865496, P124=0.02412916446778294, P125=0.013703181211705212, P126=0.002064998228134976, P127=0.0015920612180242526, P128=0.003769421746382787, P129=0.010254419316843143, P130=0.003029865140461072, P131=0.0004601910453727016, P132=0.0008714054377772468, P133=0.0029409708618251267, P134=0.0012090854425998538, P135=0.0008430969481349735, P136=0.001756175826103589, P137=0.00172232575247259, P138=0.001309840955486627, P139=0.0006135414306216931, P140=0.0006238801781727184, P141=0.0002813207762099969, P142=0.0010271867019432971, P143=0.0004906622043082662, P144=0.0003320130016749442, P145=0.0006026064512921737, P146=0.0012066555255077667, P147=0.00053987127304063, P148=0.0014787078819550592, P149=0.0023482018363807805, P150=0.0016107702985326288, P151=0.001193375369992871, P152=0.0016485749203743588, P153=0.0008115480605354489, P154=0.0007223217710936946, P155=0.0005200043973939901, P156=0.00040223800644635817, P157=0.0003431544958103363, P158=0.000326457630422365, P159=0.0003312008293928531, P160=0.00039365430064988824, P161=0.0003731282471186176, P162=0.0003584756638357031, P163=0.0005284086084568754, P164=0.0005629858867919923, P165=0.0005282221448513794, P166=0.0005305177544829792, P167=0.000506719068913751, P168=0.0005374770264930219, P169=0.0004698877614833592, P170=0.00038910195915698944, P171=0.0006414698895378774, P172=0.0005218664477409887, P173=0.0005793526157253207, P174=0.0006720239359746774, P175=0.0006135242679156416, P176=0.00038676319951178967, P177=0.0004818655217374317, P178=0.0003959495950299974, P179=5.953360222111277e-05),Row( P0=-713.1177034461263, P1=35.724528292560585, P2=-18.272790900132836, P3=-11.575598137282345, P4=-17.484835451294217, P5=-14.874394746899137, P6=-11.31004731917206, P7=-7.286835098938377, P8=-10.170796975246407, P9=-6.113866886545286, P10=-10.526377061935023, P11=-6.637656196933784, P12=-4.055181950116396, P13=-2.363069957280348, P14=-1.0189147180418854, P15=5.23028664627984, P16=5.277348937535905, P17=6.040222127124945, P18=9.25119170251767, P19=5.716463313631717, P20=4.023634845784265, P21=0.4547040667697293, P22=-2.6862740975795405, P23=-5.014509528888869, P24=-6.211119273831083, P25=-5.65351889606059, P26=-3.5577036605116756, P27=-0.2069780944304158, P28=-2.13282856303157, P29=-1.2066461838974571, P30=-1.6834488317354892, P31=0.8590544510556712, P32=-1.121756332549606, P33=2.2699149103020906, P34=-3.386475693104375, P35=-1.116863438227162, P36=-0.45736468682542863, P37=4.868615748476188, P38=3.342920532785212, P39=1.631163988914714, P40=0.648419691435205, P41=0.6248209512394877, P42=0.5784697321606316, P43=0.5699576280674706, P44=0.5617556574863822, P45=0.4981087331631851, P46=0.5250916856102117, P47=0.6071914521054884, P48=0.666257519977653, P49=0.7080238342027582, P50=0.6429911668579649, P51=0.623947488919134, P52=3.98238513604872e-07, P53=4.568157302060621e-07, P54=5.505554753351227e-07, P55=9.263073998829782e-07, P56=3.2562664816282947e-06, P57=6.525474157063791e-06, P58=3.3546528927423893e-06, P59=2.3743213825999883e-06, P60=3.075378359982858e-06, P61=6.470458876491852e-06, P62=4.540737443572703e-05, P63=0.0007915028818326218, P64=0.004058685626182556, P65=0.02454306599007376, P66=0.02603093206102489, P67=0.011402675989717751, P68=0.012642893951977105, P69=0.016021808628031275, P70=0.01164811079514026, P71=0.003730901046137635, P72=0.0003512348793963125, P73=0.000101289568029231, P74=6.322216452139405e-05, P75=0.00010827934225968423, P76=0.0004837283355864318, P77=0.0015949970169587934, P78=0.003375014001672665, P79=0.00823201049341749, P80=0.016047635939862036, P81=0.007712498661547482, P82=0.0027888286209331098, P83=0.0017367094305867998, P84=0.00362766342518628, P85=0.003828381200122247, P86=0.0045654525851293514, P87=0.004374145893345148, P88=0.007504470943652391, P89=0.007826044953529697, P90=0.004127748122487908, P91=0.0019206256626296374, P92=0.0037309023311010727, P93=0.0008597990223310407, P94=0.00011854488309476667, P95=0.000251749807422393, P96=0.0026202061302930883, P97=0.005240591171531369, P98=0.006389129779459705, P99=0.0015503820170355534, P100=0.0011732621586499669, P101=0.0009025711212172753, P102=0.00011461751986077468, P103=3.1187700994791475e-05, P104=0.0003121980814336206, P105=0.004002970786547765, P106=0.002628089763931354, P107=0.001723877224573911, P108=0.0004919287729042841, P109=5.483742058351064e-05, P110=6.887762068626824e-05, P111=0.0005303328360823604, P112=0.0027095057437957013, P113=0.0015407079406502121, P114=0.0006782419612265644, P115=0.00013931949511262912, P116=0.00037112555699763154, P117=0.0013168006907614135, P118=0.0008856661272673231, P119=0.0003603147311858029, P120=0.0002566370331834257, P121=0.0005769016187704396, P122=0.000484563377853171, P123=9.748607296466433e-05, P124=4.1512274996264745e-05, P125=5.348865154741735e-05, P126=6.378150245802365e-05, P127=2.7163864132320598e-05, P128=1.9566086340666423e-05, P129=2.2337773096995018e-05, P130=1.605382938150361e-05, P131=1.4070428541311897e-05, P132=1.0472796735189937e-05, P133=1.2888473951028473e-05, P134=2.607878809074602e-05, P135=3.3827701791702056e-05, P136=2.043054481873723e-05, P137=2.529085032315547e-05, P138=2.5642882804266663e-05, P139=3.040430876351104e-05, P140=2.1423071000558478e-05, P141=1.9913043563819826e-05, P142=7.413194979776905e-06, P143=4.902955005043372e-06, P144=5.262002194227935e-06, P145=8.159249889545706e-06, P146=6.719065741938564e-06, P147=9.52586057301399e-06, P148=9.295762628188971e-06, P149=9.781159702752686e-06, P150=1.2193327018155895e-05, P151=1.829784448973995e-05, P152=3.364607980816533e-05, P153=2.9780348095267532e-05, P154=2.915625365818271e-05, P155=4.218660158700552e-05, P156=6.160370753078578e-05, P157=0.00010456477889837734, P158=6.393580688764655e-05, P159=0.0001323302266067973, P160=0.00014583625106376235, P161=0.00012873649914770224, P162=0.0001022255354814916, P163=0.00020444575408620357, P164=0.00020359210388895112, P165=0.00018649049979203473, P166=0.00012181427310678703, P167=0.00014902235743841942, P168=0.00016536414781250617, P169=0.00010806711725352366, P170=8.535300772121442e-05, P171=4.681518706124921e-05, P172=2.9993881603631707e-05, P173=2.1290124720259102e-05, P174=2.3594159755425402e-05, P175=2.240912767115516e-05, P176=1.7981441042732927e-05, P177=1.0893642788874979e-05, P178=5.441684228906687e-06, P179=8.662954673518215e-07),Row( P0=-546.3901562530834, P1=4.928500236671237, P2=-36.240498162223346, P3=-14.567496475885104, P4=-24.69743678169255, P5=-13.365367175137273, P6=-9.605290942278696, P7=-8.336461868148126, P8=-4.812391335136076, P9=3.935225630115643, P10=9.955252545854293, P11=17.782960333046354, P12=9.473272781805626, P13=9.28369709390499, P14=-6.894179785289757, P15=-5.729257493688393, P16=-8.358553947632542, P17=-0.8216216479647138, P18=7.030813413263545, P19=-2.450463378390643, P20=-2.3387441682197143, P21=-0.8124299938212403, P22=1.3132236664422725, P23=5.238392349155357, P24=-3.735381199800523, P25=-5.6030945334709426, P26=-1.8694901538780364, P27=1.8092317019471316, P28=-1.2243108349912062, P29=0.35912760006010597, P30=-0.19345656273349862, P31=1.1131469472574091, P32=-0.8807528914239782, P33=1.0888249039242275, P34=-1.3074754326218392, P35=-0.44688690192153363, P36=-1.6880363060805295, P37=2.604545399695305, P38=-1.7409237426797959, P39=-1.9205319550601136, P40=0.5076053499338059, P41=0.5556726017893906, P42=0.520687769245341, P43=0.4815521684214878, P44=0.48797178487648546, P45=0.5052377392710761, P46=0.6027667027701145, P47=0.5947704801071, P48=0.6012223271729388, P49=0.5499906988744137, P50=0.44948271653850796, P51=0.44588925227062587, P52=3.40385880828447e-06, P53=1.2359509678932422e-05, P54=1.2266680453749305e-05, P55=1.554324852720234e-05, P56=1.536930324448418e-05, P57=1.4771229795794055e-05, P58=1.1318290690948327e-05, P59=8.245096935446393e-06, P60=6.872687543941729e-06, P61=8.054094426731421e-06, P62=1.0011327582869533e-05, P63=9.687371664986143e-06, P64=2.275481280728727e-05, P65=0.00040210983144250653, P66=0.0083359185330659, P67=0.01996863852186427, P68=0.009913491243161631, P69=0.012139087141190563, P70=0.01748483124423221, P71=0.01990243060226716, P72=0.04201104872453207, P73=0.0868800697012372, P74=0.42567384682033876, P75=0.6770590918148909, P76=0.6270641293569676, P77=0.764786019715864, P78=1.110944896587714, P79=0.7835462742312409, P80=0.17864054122003156, P81=0.021774335914159504, P82=0.010883376019627047, P83=0.012501512915598591, P84=0.012295698174385002, P85=0.008545318165465051, P86=0.00685051508262585, P87=0.0098563939339375, P88=0.012564430051895444, P89=0.012915591652118622, P90=0.01874257648031877, P91=0.03168929201352421, P92=0.035723981079504315, P93=0.13705727909262735, P94=0.45870336196670347, P95=0.984579233576102, P96=0.4674594132661779, P97=0.3316872407617419, P98=0.4343271687161526, P99=0.2479808838532735, P100=0.49157746325103746, P101=0.2407637454164211, P102=0.015370556353944256, P103=0.012041364262433999, P104=0.013062168601216013, P105=0.04253221421822262, P106=0.13816186641468106, P107=0.5198658868905811, P108=0.8602418207603524, P109=0.3356439974590193, P110=0.4374680329181888, P111=0.28788232230677024, P112=0.2396379360917541, P113=0.07708238470621132, P114=0.00536819015882734, P115=0.008800278332657454, P116=0.022765252129326924, P117=0.05841952410550411, P118=0.060158944428337253, P119=0.07361841081295196, P120=0.05180086707820907, P121=0.03280488005827781, P122=0.036526342943532825, P123=0.04382811750314581, P124=0.052451920265001124, P125=0.027153513980626962, P126=0.017086058588441116, P127=0.024287569675853982, P128=0.02594977925554458, P129=0.01879592053638992, P130=0.012766807543145204, P131=0.004669373567913847, P132=0.0035098807563979967, P133=0.0031625969645615384, P134=0.0032913291531411005, P135=0.001901454152193622, P136=0.003278952692917763, P137=0.006671969396903417, P138=0.01015422026547182, P139=0.002777261008890615, P140=0.001615190107766184, P141=0.0038800743896357515, P142=0.00261343742262815, P143=0.0004259284582830158, P144=0.0006004885043707702, P145=0.00094635039272959, P146=0.0005617747917786422, P147=0.0012017874442381044, P148=0.002150678605098943, P149=0.00328613560675998, P150=0.005128807496273834, P151=0.00404480456735395, P152=0.002160752273153775, P153=0.003671145392051275, P154=0.0018926076300667392, P155=0.001033108469135934, P156=0.0006509149781781003, P157=0.00059007480086985, P158=0.0008279316852104121, P159=0.0012467399029199426, P160=0.0014946546693596653, P161=0.002885948041606584, P162=0.002965614097978421, P163=0.0026929640321825387, P164=0.002367433911592862, P165=0.0023250160661968423, P166=0.0024701364701349, P167=0.0022396847542015662, P168=0.0019662727408399773, P169=0.0018149460703358817, P170=0.0016793545134577272, P171=0.001865402023526554, P172=0.0012907991499094575, P173=0.0009904655412427499, P174=0.000889111315107718, P175=0.0006533378848817399, P176=0.0005615397778277487, P177=0.00032127527316240165, P178=0.00013921805830285176, P179=1.627179097017047e-05),Row( P0=-635.9011627512298, P1=45.26455330033845, P2=-32.08974030550214, P3=-18.16890958262293, P4=-26.714816218052952, P5=-7.55851609022641, P6=-12.140234090029171, P7=-3.309129683152808, P8=-2.1474958905473693, P9=-9.496974463168605, P10=-8.952993403759363, P11=-0.6955950678725877, P12=-9.106553376449304, P13=-0.38974747696975004, P14=-4.07387692453359, P15=-1.2427455903184725, P16=-0.5599960632707087, P17=-4.326400380413556, P18=-0.48224751950834416, P19=-1.3793197102795658, P20=-1.6581245491565355, P21=0.422911289999296, P22=-2.789699539888155, P23=-1.7769125412532794, P24=-4.153526617209588, P25=-3.7320778304349296, P26=-2.5644057559614986, P27=-0.08673632347112883, P28=0.07401239809071797, P29=1.751472976971473, P30=-0.7229535438936878, P31=1.33718649309932, P32=-1.011516134791386, P33=0.19288391127955942, P34=-1.462506707582922, P35=-0.38873775837880753, P36=0.6935864888950823, P37=0.29127435472627433, P38=-0.2635159487949242, P39=0.8526457930990474, P40=0.7037508153792856, P41=0.6948672717285465, P42=0.7201534839300221, P43=0.6597413926792483, P44=0.6495078818415068, P45=0.6356203049509037, P46=0.6555959587098493, P47=0.6702877165692568, P48=0.7044508031996017, P49=0.71811167906344, P50=0.6863038999465987, P51=0.6694784238012816, P52=1.4002691634704868e-06, P53=1.1988668954717526e-06, P54=8.73884229487253e-07, P55=3.523554649723076e-06, P56=1.6618314696178678e-05, P57=4.8588771785513214e-05, P58=3.148076065422702e-05, P59=3.0001477063556668e-05, P60=5.116508993447377e-05, P61=9.729590221750297e-05, P62=0.00014845548479622728, P63=0.00017763750488685115, P64=0.0002119519510217962, P65=0.0007794556688134702, P66=0.002248681967367669, P67=0.0035789639778245627, P68=0.003934818999674868, P69=0.010463594676722861, P70=0.012230384685375906, P71=0.003433767377206555, P72=0.0004557985591030676, P73=0.0002713189507637295, P74=0.0005443303549878302, P75=0.0008184888937737289, P76=0.0010950684362948536, P77=0.002486419004350327, P78=0.002946234684020967, P79=0.0022852519585520614, P80=0.0016707133220123, P81=0.0011489268722103194, P82=0.0009003819879564543, P83=0.001071281433151146, P84=0.0022261977634173515, P85=0.0031358033051984173, P86=0.011000126538203103, P87=0.06870034542715509, P88=0.06649268813905419, P89=0.016791346253381335, P90=0.005734770677371799, P91=0.004094877685401929, P92=0.004257777665353837, P93=0.002578004220343943, P94=0.006481562796666772, P95=0.01760753646561402, P96=0.009600073784692274, P97=0.008645403898342983, P98=0.010048124670102096, P99=0.00530812698216262, P100=0.005846262366263786, P101=0.005615092299334745, P102=0.003429846668097645, P103=0.0025586665423236532, P104=0.002281914683158338, P105=0.005415570357128876, P106=0.00334128663482083, P107=0.002073640776348215, P108=0.0015377952707956635, P109=0.002282600127957045, P110=0.0036486846778785564, P111=0.0025665328858829805, P112=0.003693827958736955, P113=0.003568507591269966, P114=0.0026311785694795257, P115=0.0008916215326596738, P116=0.0009352737714395325, P117=0.0018972982434948805, P118=0.004111959778430207, P119=0.0009343589289273238, P120=0.0012722000377972708, P121=0.0010964169494588932, P122=0.000494308835758735, P123=0.00018231664305521215, P124=9.62259228923737e-05, P125=0.00014737450359745004, P126=0.0001576370485961195, P127=5.760094213491297e-05, P128=6.863661319593337e-05, P129=0.0001952333044272664, P130=0.00043375585564639924, P131=0.00018427285298882234, P132=9.758107859135486e-05, P133=0.00014579815427864361, P134=0.0001779843903136695, P135=8.601928662114075e-05, P136=0.0002322624898389037, P137=0.0004324977115218307, P138=0.000177103225202418, P139=0.000659161548315884, P140=0.0011691831003813353, P141=0.0001922329751423642, P142=0.0002604804768230161, P143=0.000358180357094476, P144=0.00036668461446225727, P145=0.00046927112400998154, P146=0.0002551191389346214, P147=0.00037962321673108256, P148=0.00022108223549307295, P149=0.0003207873776947862, P150=0.0001906007819026177, P151=0.00017739260138448747, P152=0.00010790365725179449, P153=7.503477538292482e-05, P154=5.460902795438269e-05, P155=9.271420866953684e-05, P156=0.0001916873580391325, P157=0.00018191688341415396, P158=9.532780720476083e-05, P159=7.354839193624102e-05, P160=7.514176377987206e-05, P161=8.239213472241474e-05, P162=0.00011686899073947217, P163=0.00012772284974875412, P164=0.0001384039948927512, P165=0.00010279640312921534, P166=8.569616229289435e-05, P167=7.675961029973356e-05, P168=7.460453606752108e-05, P169=7.778683272518064e-05, P170=6.694662589924788e-05, P171=6.2680400053321e-05, P172=3.4465677552842834e-05, P173=2.6092654603093644e-05, P174=1.7385988972634994e-05, P175=1.1455538264160324e-05, P176=8.62239634897083e-06, P177=1.0507879937684067e-05, P178=5.446120583293551e-06, P179=7.235756760787921e-07),Row( P0=-601.3287973460392, P1=36.67823506615112, P2=-36.14093394508706, P3=-8.203149273344225, P4=-25.492242906694532, P5=-10.38620718796601, P6=-7.752344798631779, P7=-7.554821610863805, P8=-5.639221775928078, P9=-11.22639604434455, P10=-8.819581391222469, P11=-3.093108947997242, P12=-9.83957355702613, P13=-0.37548451784758136, P14=-0.3142061590480257, P15=2.2685828380418105, P16=5.209861965513148, P17=1.3692481364927138, P18=3.144481736638341, P19=2.616366086302248, P20=-1.433769113205317, P21=1.3592508565444288, P22=-1.3002097753940782, P23=-0.2830658375560712, P24=-3.1816705850873253, P25=-3.73665528420478, P26=-4.6523489476949305, P27=-1.1851487404526642, P28=-0.2861556561116045, P29=2.032971880570578, P30=-1.1127395443637593, P31=0.23639501267346488, P32=0.041793653943840846, P33=2.285613453931281, P34=-0.23699874238949115, P35=-2.2082556951456587, P36=-2.681953232434975, P37=0.33044608380415197, P38=1.7888340006245285, P39=2.505401663222949, P40=0.6190636012595255, P41=0.6477771111597299, P42=0.5649518845188995, P43=0.5257976226671147, P44=0.5164838293402528, P45=0.4812513804342861, P46=0.48826836494575443, P47=0.5315057580756009, P48=0.584209642955652, P49=0.6303385737425006, P50=0.5929543227873507, P51=0.5868361178805597, P52=1.561079938066399e-06, P53=2.8532026970425198e-06, P54=5.320836689067296e-06, P55=7.086187372607661e-06, P56=1.861409833639707e-05, P57=3.920718149469832e-05, P58=1.7667091256081663e-05, P59=1.5485874771254194e-05, P60=3.9381280115666144e-05, P61=8.973190969874187e-05, P62=0.0005815461377622506, P63=0.0022941788982460813, P64=0.007681188456149103, P65=0.015272082321225821, P66=0.02622298919014201, P67=0.0403949308335752, P68=0.04398424286414201, P69=0.03002478874213252, P70=0.01923309173238191, P71=0.028813331541929676, P72=0.010797836240014928, P73=0.002901859601725027, P74=0.0020319089476772707, P75=0.0035533806552264747, P76=0.004757136146429185, P77=0.014945566720217171, P78=0.03131174916500547, P79=0.02638385770725969, P80=0.006880724343758139, P81=0.007580601067088612, P82=0.022309070544643747, P83=0.03488778642618915, P84=0.07220709973057839, P85=0.057848147981952126, P86=0.021393159512662567, P87=0.026061130566762295, P88=0.04858920312807222, P89=0.050379462977483504, P90=0.04226610999690858, P91=0.03335023272557697, P92=0.006015551410036691, P93=0.005178443815902648, P94=0.01805675688090689, P95=0.02560575066293347, P96=0.03559066285785864, P97=0.10346183234331223, P98=0.038087992506950885, P99=0.03811625625342146, P100=0.03833396513088102, P101=0.004665726395081694, P102=0.0065475077354404, P103=0.005812826963566373, P104=0.004138438660739948, P105=0.0798626081147252, P106=0.13812403443556745, P107=0.03325422679961895, P108=0.007860321865202654, P109=0.0018179935107357, P110=0.0028889352084818854, P111=0.00386670331065273, P112=0.020395167491333722, P113=0.02348006765567473, P114=0.006427287978335301, P115=0.003090187591812894, P116=0.004027298797011278, P117=0.008411304089241591, P118=0.004463531404605913, P119=0.002812612671536878, P120=0.0029242650653532615, P121=0.006232028214443497, P122=0.001164419203687699, P123=0.0033879893648303943, P124=0.0029651940449797153, P125=0.0035272668027389697, P126=0.00021405255596751086, P127=0.0008537016118764398, P128=0.0019164273454883242, P129=0.0004631574033688769, P130=0.0005406938044891083, P131=0.0014922725534488644, P132=0.0012193261303388994, P133=0.0011666916131723205, P134=0.000879008361216642, P135=0.0005688342756790553, P136=0.0012828771842334262, P137=0.0005225297752559769, P138=0.0005580639845218569, P139=0.0005009798836061661, P140=0.000806595464092225, P141=0.0006887201330162276, P142=0.0005233055684124386, P143=0.0003702581984288701, P144=0.0006045757335728906, P145=0.00037082545935714995, P146=0.0006731377552634471, P147=0.00041185964036723644, P148=0.0009386136687666864, P149=0.0010282772374402914, P150=0.0010540901716361907, P151=0.0005474297137413662, P152=0.00019333417560133605, P153=0.00014267065873418662, P154=8.656876508775983e-05, P155=6.980550747629253e-05, P156=5.541205983841936e-05, P157=0.00010380092113514005, P158=0.00013644584898600583, P159=0.00010182326028791132, P160=8.829855558013316e-05, P161=0.00011644890981395037, P162=0.00013521148775149692, P163=9.761302545569151e-05, P164=7.906973186894114e-05, P165=3.830151133066522e-05, P166=4.8531204337059704e-05, P167=4.57768511449664e-05, P168=5.412026970874781e-05, P169=9.99860507970721e-05, P170=0.00010644887951885572, P171=0.00010337913031832041, P172=0.00010084073496802687, P173=0.00013295136888802545, P174=0.00013461197501345073, P175=8.93487590143501e-05, P176=3.872478020843711e-05, P177=6.219773626243194e-05, P178=3.231627295856307e-05, P179=2.7479968117016043e-06),Row( P0=-760.4511225382507, P1=36.593245317706966, P2=-9.914774060847927, P3=-7.223111854561318, P4=-20.713349633457455, P5=-9.845209280621535, P6=-10.913900310355592, P7=-1.9296549096520128, P8=-5.2997284155513915, P9=-5.6903835321480125, P10=-6.3906108582071335, P11=-6.154873152318022, P12=-6.931341769829564, P13=-4.366934588073332, P14=-4.302168273531532, P15=6.681127384588181, P16=7.799427705139441, P17=14.15002948905927, P18=15.634336980716537, P19=13.926969340138314, P20=4.767231797696151, P21=0.4267334557354833, P22=-6.247208355407639, P23=-7.685730488376472, P24=-2.414851931943147, P25=-2.3257545607481203, P26=-2.4426791246544917, P27=-4.481531308367743, P28=-7.806835785178102, P29=-5.264032305518076, P30=-1.0097103545905008, P31=1.2933885536172287, P32=-1.0922380945856942, P33=-1.355097731273493, P34=-1.0579821220317975, P35=5.062063918427111, P36=7.183503632446571, P37=6.773079116071084, P38=-0.12495157016921023, P39=-0.51036967676209, P40=0.6194114626283653, P41=0.5458262475112917, P42=0.5256273006569933, P43=0.4890050557857482, P44=0.49230852534865943, P45=0.5635543231982841, P46=0.5576898844647867, P47=0.5119955008652399, P48=0.5468610160794711, P49=0.6536777292577085, P50=0.8488588203706053, P51=0.7786925450712578, P52=5.62160023355011e-07, P53=9.67610944246955e-07, P54=9.788642791843035e-07, P55=6.921021708557131e-07, P56=1.9393562699774448e-06, P57=2.8313809367413702e-06, P58=1.103521555441777e-06, P59=5.215087516247805e-07, P60=8.366780449307638e-07, P61=2.8606892511785377e-06, P62=9.94883215424578e-06, P63=3.997760895564407e-05, P64=0.0006394522218440878, P65=0.01861569542448719, P66=0.03013751469900704, P67=0.012837878573944419, P68=0.001942285453517778, P69=0.00023762492156734978, P70=0.0001027266427864814, P71=2.972976285874514e-05, P72=2.0139494432291247e-05, P73=1.5083720206816192e-05, P74=8.629508523170665e-06, P75=1.0735201783466526e-05, P76=1.5471908708324616e-05, P77=5.107061744202932e-05, P78=0.0002606827625965442, P79=0.0024741199636701696, P80=0.007575694502684723, P81=0.0060731925327609125, P82=0.00441305355752837, P83=0.0012836368942278043, P84=0.0006142107891873704, P85=0.00041609375005386643, P86=0.0001242326353106338, P87=7.501261483648731e-05, P88=6.222002701359669e-05, P89=3.3244544381062346e-05, P90=3.0465347994236008e-05, P91=0.00011735962433766615, P92=0.001842345140418214, P93=0.0020114238729036253, P94=0.006640809252222554, P95=0.0036396836774227515, P96=0.000672842235433261, P97=8.359957682916468e-05, P98=2.875308925443436e-05, P99=1.0590177148305243e-05, P100=7.63757461104723e-05, P101=0.0016271119937895876, P102=0.0009620236359468142, P103=0.0015298774069930888, P104=0.00020437535158974055, P105=2.2244690912104146e-05, P106=1.1061003883004357e-05, P107=5.4092754003092184e-05, P108=0.0005419667319658639, P109=0.00016432297861860686, P110=0.00022204169273692877, P111=5.706406132666819e-05, P112=9.524470656082417e-06, P113=3.160598319316914e-05, P114=8.68256225330187e-05, P115=0.00013274595325470868, P116=0.00020496852145032393, P117=1.6917414022172256e-05, P118=8.40379387094818e-05, P119=7.884577979102692e-05, P120=3.1094437222649765e-05, P121=2.423085792942222e-05, P122=6.378287748697004e-05, P123=0.00011084692176703076, P124=2.062622930494845e-05, P125=1.4512761902072441e-05, P126=1.8277952298121986e-05, P127=1.0340078056602998e-05, P128=1.001204221806387e-05, P129=8.224361749298261e-06, P130=3.6781943375843522e-06, P131=2.7458390580764884e-06, P132=4.51015073290718e-06, P133=4.996243349483587e-06, P134=9.48618619277225e-06, P135=2.0958100071338916e-05, P136=8.443201395908646e-06, P137=2.0774985035914077e-05, P138=1.3661676567689494e-05, P139=1.562480915227529e-05, P140=6.731161701057412e-06, P141=2.8941490528304692e-06, P142=2.291501777816853e-06, P143=2.299016594816991e-06, P144=3.712491424609799e-06, P145=8.47823474157005e-06, P146=1.3483218510939744e-05, P147=1.317109168242855e-05, P148=1.599494299493242e-05, P149=1.4954790073225338e-05, P150=4.063982451799276e-05, P151=4.32763030941954e-05, P152=2.8309852944329088e-05, P153=2.9954403597704334e-05, P154=2.8151026218886503e-05, P155=4.4903579798852464e-05, P156=0.00014639772659125644, P157=0.0001380514042582559, P158=7.582065746061877e-05, P159=9.288007446722688e-05, P160=0.0001311544004589978, P161=0.00010981835537830557, P162=0.00012906397102486724, P163=0.000158259692881879, P164=0.00013109509012479642, P165=8.826620534418696e-05, P166=4.481108153916532e-05, P167=4.513624769116373e-05, P168=3.3737870871166595e-05, P169=3.644581935747616e-05, P170=3.3343173860853146e-05, P171=2.7191358184509292e-05, P172=1.7527410991696703e-05, P173=1.7235959531430596e-05, P174=1.2358507760663225e-05, P175=1.0886829069724533e-05, P176=8.693602442181959e-06, P177=6.116819647521095e-06, P178=2.1793760162834008e-06, P179=2.1379829086694792e-07),Row( P0=-493.3035756000161, P1=-10.906560142116758, P2=-38.888516310778854, P3=-17.53951641699621, P4=-18.228371280728705, P5=-8.391072304672058, P6=7.666156839228965, P7=18.635105296025852, P8=12.805863837203944, P9=-1.1102921855601113, P10=-11.998798943619482, P11=-4.907691210679835, P12=-4.543374740911345, P13=8.454485935991505, P14=2.529071526228513, P15=-2.1476972610068708, P16=-6.145299655377318, P17=-0.32577714525922996, P18=4.514894794735448, P19=-2.247785391909115, P20=-1.6160951982320346, P21=5.667688723794376, P22=-0.45216878548403555, P23=-3.5916641169485004, P24=-2.650027377591621, P25=0.5420395382367109, P26=0.32189740996343263, P27=-1.2815661793445232, P28=-1.18591513298657, P29=0.9600762098549337, P30=-0.8153042009648778, P31=-0.08260005839681493, P32=-0.855959178228483, P33=0.6703344688125503, P34=-2.026649517858303, P35=-1.5278362889064807, P36=-1.6581812150636666, P37=0.07454133322690917, P38=0.3632271897771561, P39=-0.33094188405466757, P40=0.5106943250274675, P41=0.5393853308478008, P42=0.5467360045981173, P43=0.555484611367439, P44=0.5588525030185796, P45=0.45998668785880514, P46=0.3819229522708515, P47=0.37175665386846246, P48=0.4010555226599096, P49=0.43513721084806944, P50=0.45644839140682536, P51=0.47167524002612526, P52=9.630473583582068e-07, P53=3.995186044092269e-06, P54=1.1176017137075146e-05, P55=2.4675877161713862e-05, P56=2.6455556444854613e-05, P57=1.9636387998571077e-05, P58=1.0887372253869542e-05, P59=6.489286432697886e-06, P60=9.665799516031413e-06, P61=1.812380870035303e-05, P62=2.7769335561375556e-05, P63=2.0113303397229956e-05, P64=9.961059700531275e-06, P65=1.5289866337124884e-05, P66=1.717896561748764e-05, P67=1.7399646582269514e-05, P68=1.824056568922243e-05, P69=2.8107077898719736e-05, P70=2.8732092603830242e-05, P71=2.2585433531280368e-05, P72=3.077174598799308e-05, P73=5.279428491930004e-05, P74=0.00031462313739154854, P75=0.0008354171524713322, P76=0.004517615848666218, P77=0.052993686339291295, P78=0.14738264542903567, P79=0.20187139336135146, P80=0.18839298435019344, P81=0.10566892681629671, P82=0.07048491637798221, P83=0.07612958326115364, P84=0.18833212574646607, P85=0.4202456058593749, P86=0.8647965483793607, P87=1.4234785052920984, P88=2.0289228939962913, P89=3.252185381250563, P90=4.408014231246727, P91=4.722262697211809, P92=4.745233743997729, P93=6.2027732279637755, P94=5.5739031596025255, P95=5.860167475719428, P96=2.674668794418621, P97=0.7295779531961394, P98=0.08634232495616088, P99=0.03387404134943353, P100=0.04942272194174654, P101=0.028890192518054834, P102=0.007064596970353006, P103=0.008581289859911285, P104=0.019824259055475954, P105=0.05847011271205856, P106=0.06292776621613211, P107=0.06445700426464161, P108=0.07855389263469019, P109=0.12518719967923828, P110=0.15736153963448715, P111=0.2269598770668628, P112=0.16983613420493837, P113=0.4200415156055227, P114=1.1428282835702135, P115=0.43372212358535794, P116=0.17584338757039109, P117=0.18168954320988842, P118=0.1453228216639173, P119=0.008848914157616359, P120=0.017368849665157136, P121=0.10932628971829768, P122=0.1547680937041253, P123=0.26135649768877356, P124=0.45547299307297034, P125=0.11594253227182431, P126=0.3905112917598459, P127=0.4424902895991964, P128=0.2547235397093991, P129=0.009564097894098317, P130=0.004900882194249391, P131=0.0034378188797368828, P132=0.005211010073218534, P133=0.011713135759461743, P134=0.018192717035354026, P135=0.05908357995986392, P136=0.02012570093034736, P137=0.019746234258379562, P138=0.022529327624362543, P139=0.019874303202738235, P140=0.004966203363314368, P141=0.004726088797756287, P142=0.005974191246765218, P143=0.0030955274291874254, P144=0.005048904874637524, P145=0.020246692057321145, P146=0.0188789693333352, P147=0.02096097257381053, P148=0.016717829209888038, P149=0.00764978082545615, P150=0.00842493027632855, P151=0.004948506984650607, P152=0.0033572324624146, P153=0.003741295266359982, P154=0.0018101899581213306, P155=0.001083493161709959, P156=0.0013161365557039346, P157=0.0015383380592791408, P158=0.0016825364432726281, P159=0.0019067691378143513, P160=0.0020549052339520766, P161=0.00358792291578898, P162=0.004502039847038024, P163=0.0051922606545860206, P164=0.004827894082154178, P165=0.006054580128769584, P166=0.007538506793270701, P167=0.004942675041652782, P168=0.004091855937355679, P169=0.003871706262898784, P170=0.003130625532658444, P171=0.0026786394517816295, P172=0.0023221522967118258, P173=0.0016522098416912052, P174=0.0015347165724153215, P175=0.0014546870786616324, P176=0.001424359829199729, P177=0.0008982206700810947, P178=0.0004169850968089166, P179=5.863418770473824e-05),Row( P0=-620.0776412275129, P1=28.338497458308378, P2=-36.87489315643032, P3=-9.257798578904346, P4=-31.965192455499846, P5=-12.614614928790726, P6=-12.05980363240199, P7=-4.860196852857096, P8=-5.119717603919767, P9=-10.900233010601696, P10=-7.903342428581298, P11=-5.611806305418522, P12=-9.940949551177376, P13=0.08396598983607723, P14=-4.176118049794483, P15=3.5368967846819066, P16=8.909740427020512, P17=5.9426903620626845, P18=10.316295976665886, P19=2.890310317788609, P20=0.9846093520260644, P21=0.7952332117398898, P22=-1.0823889706268532, P23=1.4820065712621537, P24=-3.432064694926063, P25=-4.476987137701298, P26=-6.545909131269207, P27=-2.161857902948109, P28=-0.5147300246406833, P29=2.6303017227032033, P30=0.11894280792295357, P31=0.6638231248995836, P32=-1.1182070172916216, P33=4.343829698500714, P34=1.3191079228156577, P35=1.593879889691162, P36=-2.44896157954698, P37=0.17657481218288512, P38=0.8417319650027691, P39=1.3066036511645922, P40=0.6484823658132609, P41=0.6399830991932328, P42=0.5952136433156977, P43=0.5750303121563242, P44=0.5295729576095134, P45=0.5016609773854935, P46=0.477266255543751, P47=0.5252660469289405, P48=0.6007588301715701, P49=0.5900196049320928, P50=0.5976133833243031, P51=0.6297786739683354, P52=3.6791527875330986e-07, P53=9.181800693664202e-07, P54=8.19892222802525e-07, P55=5.590657044369318e-07, P56=2.136181875567798e-06, P57=6.444269572164357e-06, P58=1.2619849595239642e-05, P59=1.9667085057878716e-05, P60=3.8089939559229715e-05, P61=4.828807419575956e-05, P62=0.00018259323814923497, P63=0.0009885918445031523, P64=0.00248071493321887, P65=0.007492919952069783, P66=0.02361777147759629, P67=0.03659180239195476, P68=0.04492027105656733, P69=0.023770482683597643, P70=0.0033536098376985225, P71=0.0004359661471286444, P72=0.0002653427567378462, P73=0.00030872646215650377, P74=0.00039605509814294183, P75=0.0012133272569506246, P76=0.0046395145028680215, P77=0.006880729950873532, P78=0.008336422444847964, P79=0.008413878508786445, P80=0.018355794704158497, P81=0.029268113848136924, P82=0.03152360781904107, P83=0.043945506491941566, P84=0.09524179075111269, P85=0.11127999874861087, P86=0.0481183121574527, P87=0.01965814474136964, P88=0.02957663586829776, P89=0.022897026976877198, P90=0.016750810793584422, P91=0.01270596348557235, P92=0.007201517475218528, P93=0.005424122037402636, P94=0.008534949416828534, P95=0.05265451625331263, P96=0.05292067075701638, P97=0.195012823720802, P98=0.12816631301661036, P99=0.0070483172915928845, P100=0.0014476242409848822, P101=0.0007086513872046621, P102=0.0009981938974818697, P103=0.007136417827925764, P104=0.01864006496235423, P105=0.035070897994270524, P106=0.09185422183150078, P107=0.027685599060337882, P108=0.00139974315910533, P109=0.004777483372328534, P110=0.0066401008499586255, P111=0.014936108285236496, P112=0.014875644909659678, P113=0.08211983048012811, P114=0.018032883785195693, P115=0.0017955719274997, P116=0.0008243493731627463, P117=0.0015972264190362822, P118=0.008657997725654062, P119=0.007828042391197745, P120=0.0012727710086332293, P121=0.0017403468831077638, P122=0.0024815993246986977, P123=0.001544519806494075, P124=0.0016512530761581712, P125=0.001305092623516718, P126=0.0010498802804182173, P127=0.0013425004330006192, P128=0.0004816049912718077, P129=0.0004038240676221919, P130=0.0010525206481988016, P131=0.0007799355240935059, P132=0.0003723403254453054, P133=0.00082324682439548, P134=0.001321308616699601, P135=0.0005757327038225779, P136=0.0006002047014354006, P137=0.0013960373271401688, P138=0.0007846626434407102, P139=0.0007141471198734827, P140=0.0005569807343141091, P141=0.0013686962077724342, P142=0.0004340626049468528, P143=0.00028922775391360226, P144=0.0005842075016768694, P145=0.0003544681245820545, P146=0.0006390820644608683, P147=0.0006932306296593495, P148=0.0007271126503839939, P149=0.00106620637827479, P150=0.0015391900707054265, P151=0.0010480857280212954, P152=0.00137165064658655, P153=0.00042218102853078477, P154=0.00027974003967917233, P155=0.0001136234082559706, P156=0.0001067345645972145, P157=0.00010710350283608681, P158=0.00014127690864255502, P159=0.00010605650618796482, P160=0.00014016426998794047, P161=0.00018529391616631003, P162=0.00017674688790581208, P163=0.00021173633678076137, P164=0.00027838488832583423, P165=0.00028928063734588657, P166=0.0003168214690714429, P167=0.0005565208886946689, P168=0.0004972199981954582, P169=0.0007379338820067118, P170=0.0005515584371468912, P171=0.0002804743531251407, P172=0.00020621333850981356, P173=0.00024365569215863614, P174=0.00023875131003355953, P175=0.0002570249273830486, P176=0.00020402053859785177, P177=0.00014646168205597856, P178=4.7726805075095683e-05, P179=4.658848758239075e-06),Row( P0=-597.0868889525335, P1=41.05033036826833, P2=-35.433588495364994, P3=-11.369373975640583, P4=-30.882402565918063, P5=-16.711226020447018, P6=-11.951128298019704, P7=-5.960198591225639, P8=-5.520342972368899, P9=-9.497282103772536, P10=-12.100564062928754, P11=-3.6941737623670563, P12=-9.683326561412569, P13=-6.175260217251866, P14=-3.5679283226147533, P15=-5.199396124894788, P16=3.2938232258735707, P17=3.9751237955840235, P18=5.838507643564849, P19=5.521616345126406, P20=1.8052847343229024, P21=2.9251334468216856, P22=-3.3445556969315873, P23=-1.1366869705753428, P24=-2.1829810952998248, P25=-1.0352927219621815, P26=-1.5470428566109624, P27=-2.159195725152932, P28=-2.0338347961368086, P29=-0.08800973241529354, P30=-0.736452342125342, P31=1.824548720665763, P32=0.9905237899971627, P33=1.3537921213050248, P34=0.47476525598296393, P35=0.9721176681378115, P36=0.5492804964301318, P37=1.1318264060967684, P38=0.8615567857332175, P39=-0.8914274420714828, P40=0.6317093101441906, P41=0.5856093489593235, P42=0.5606470724361002, P43=0.564542232646847, P44=0.6077354572106451, P45=0.5756086413302994, P46=0.5779495685941957, P47=0.6017837783717813, P48=0.6542837304458006, P49=0.6645501841208127, P50=0.6471513415985697, P51=0.64573095393115, P52=4.713753749862878e-07, P53=5.531988033985484e-07, P54=6.178982083518409e-07, P55=1.5554877211958448e-06, P56=5.998362383754968e-06, P57=1.3877151394484361e-05, P58=2.1626437890066627e-05, P59=3.277358670781896e-05, P60=4.8031139103092876e-05, P61=0.0002166665573633754, P62=0.0004910076040310012, P63=0.0009656107198644632, P64=0.004635797872018396, P65=0.02408209366612877, P66=0.04930811322610984, P67=0.02711790340855478, P68=0.010688514865277143, P69=0.009520442471235672, P70=0.003327939991571108, P71=0.0007631060459493444, P72=0.0006026719900091193, P73=0.0005714300664659473, P74=0.001169405623531806, P75=0.0015861943040595947, P76=0.0018381675385921936, P77=0.00675847464255874, P78=0.018920675197840787, P79=0.03708043046270584, P80=0.10340208261348373, P81=0.11907337064230675, P82=0.07880749735526393, P83=0.037577419240148305, P84=0.05712160744137293, P85=0.0661012997082109, P86=0.0250234682015662, P87=0.01732008048462932, P88=0.012277762278041547, P89=0.010163317228497868, P90=0.025992136860818606, P91=0.03892949293725398, P92=0.02777114495078379, P93=0.04235159446566866, P94=0.069780956352649, P95=0.06859283931976373, P96=0.08913892677791489, P97=0.12753773137547417, P98=0.04553270200396867, P99=0.06623651775960387, P100=0.09192674375623974, P101=0.02828977006159241, P102=0.01452654928766483, P103=0.016083326463636194, P104=0.021718591666778905, P105=0.03810748165494522, P106=0.026328195641191366, P107=0.012053562336259115, P108=0.005232155906644234, P109=0.002961856545416355, P110=0.004596430716214718, P111=0.009070784241739616, P112=0.06638393344236528, P113=0.060983359921005724, P114=0.01340494371556334, P115=0.00989530536174728, P116=0.010920014947084474, P117=0.014327535193993783, P118=0.003922916403537543, P119=0.0015159225756411002, P120=0.00636053393369112, P121=0.00904925211437817, P122=0.0015833453746769927, P123=0.003261070216146868, P124=0.00861003124165462, P125=0.004815894348393451, P126=0.0007026682082231445, P127=0.0022899918791245712, P128=0.002575389122238132, P129=0.0002645635139705604, P130=0.002349728808256622, P131=0.0023504425593990036, P132=0.00035390452983534204, P133=0.002214785026487318, P134=0.0014669218929738404, P135=0.0010632102854808993, P136=0.001209329280687845, P137=0.0007507848956358963, P138=0.0013607887114682907, P139=0.0030697677631411836, P140=0.0011622007650506495, P141=0.0014754442321763942, P142=0.001408531308724542, P143=0.0011552967487115995, P144=0.0011139941299513692, P145=0.0013409676140994, P146=0.001101227071147897, P147=0.0008688524552719373, P148=0.000963514875147527, P149=0.0005515021273176142, P150=0.0003144924550827626, P151=0.00018023425645572076, P152=0.00010881129111060052, P153=0.00012886374890456293, P154=9.728819166480118e-05, P155=0.0001763784506265603, P156=0.00010808847788969108, P157=0.00015651945239037354, P158=0.0002268974260049761, P159=0.00041132432589578174, P160=0.0003691052413994098, P161=0.0007156112531217485, P162=0.0006863318067315328, P163=0.0008311809980043064, P164=0.0007749029709678185, P165=0.0007617482480314263, P166=0.0007255572740248996, P167=0.00041636688775964324, P168=0.00047816566732734075, P169=0.0007418256408844703, P170=0.0006072227700268306, P171=0.000514295151835267, P172=0.00043821850032704253, P173=0.00042627996310303786, P174=0.0003549821301896613, P175=0.00021757382150699287, P176=0.00012084263696923794, P177=0.00011429998388367735, P178=0.00011521083545826667, P179=1.8838867630766604e-05),Row( P0=-648.0421400634851, P1=22.767846188844967, P2=-30.257858260956482, P3=-9.025217916847547, P4=-21.48604109097664, P5=-14.32984332287356, P6=-10.705908802538705, P7=-6.09971613892161, P8=-10.697995718676756, P9=-7.971079633365314, P10=-8.404123344697183, P11=-5.596061747292927, P12=-2.7091677019664426, P13=4.15326653186793, P14=0.8466800457026633, P15=8.015014886312647, P16=5.672226872543178, P17=2.9218315293496397, P18=6.0129167294337975, P19=0.6564973047114719, P20=1.5824460635523923, P21=0.7505429632659241, P22=-4.355592342573516, P23=-2.6306925073470553, P24=-5.866638292311151, P25=-4.557825672384282, P26=-1.77543037600663, P27=-0.5347757608498841, P28=-1.0495278803147203, P29=-1.1981248111001128, P30=-2.5302532503796087, P31=1.356220704330696, P32=0.7722764046991091, P33=2.5701416573202933, P34=-2.9316998067838367, P35=-2.732614878013019, P36=0.2221603983391668, P37=4.4264577709790265, P38=2.861584318709317, P39=-2.4336406833376207, P40=0.5786638252522033, P41=0.6132500521890252, P42=0.5510657820543258, P43=0.5125523291912226, P44=0.5565091329478089, P45=0.5051008604878795, P46=0.4672529392126327, P47=0.49419476275086777, P48=0.5428090929046295, P49=0.6116360760073238, P50=0.5640735049297658, P51=0.5099176518823454, P52=1.744388235755319e-07, P53=3.373032596275981e-07, P54=8.910454320490063e-07, P55=1.4130038383985703e-06, P56=2.6986286705990597e-06, P57=3.6805120784943913e-06, P58=2.4144705562357033e-06, P59=2.353284864424372e-06, P60=3.559020083143323e-06, P61=4.753308119087101e-06, P62=1.9373258926411747e-05, P63=0.0002213708757957165, P64=0.0019072365132430577, P65=0.014038698251667252, P66=0.020377831872613567, P67=0.020309546274745828, P68=0.04262806529329978, P69=0.037402544016149376, P70=0.020005565936223536, P71=0.008762289835577263, P72=0.005852802597616102, P73=0.008849807575780015, P74=0.008411724365970579, P75=0.0017020378653867184, P76=0.0003892133055872018, P77=0.003276037776698875, P78=0.0070679071425266814, P79=0.008452758094814857, P80=0.014079047410319765, P81=0.015883412315821913, P82=0.009928986267638747, P83=0.00876511052720777, P84=0.04642924834736781, P85=0.05786347702962952, P86=0.03205516236472994, P87=0.022597625742003653, P88=0.025641400193378648, P89=0.013623666305892361, P90=0.01116837940423637, P91=0.013355897759173764, P92=0.043129774667551406, P93=0.014390939330099516, P94=0.005165136114972072, P95=0.0025526396120196587, P96=0.0009011209376408024, P97=0.02407300882321182, P98=0.09109583461485345, P99=0.0448758672744419, P100=0.13207244670348278, P101=0.08085746013416226, P102=0.005221571901486061, P103=0.0012404713506449338, P104=0.00036573288203175066, P105=0.0010594683003877145, P106=0.01611312657039307, P107=0.09387247293493381, P108=0.06835353757618694, P109=0.031307696851466844, P110=0.00359748436665283, P111=0.0005218238996437182, P112=0.001911841980220238, P113=0.018386066829848585, P114=0.013080650088850433, P115=0.011425870472878684, P116=0.03182560169246016, P117=0.013535583651458509, P118=0.017266988986393134, P119=0.00436085357736431, P120=0.0009437476131395713, P121=0.0024672929322290134, P122=0.007211206772864247, P123=0.0011087428324189246, P124=0.0002737604199196441, P125=0.00075565777021905, P126=0.0011798295207858108, P127=0.00047702832999036967, P128=0.0003094424733857209, P129=0.0003144206436161836, P130=0.00020125031170989302, P131=0.000341599386834311, P132=0.00032604508060680336, P133=0.00016287533442658816, P134=0.0009690389649847361, P135=0.0006446939600818686, P136=0.0004716053197163998, P137=0.0007119092341153067, P138=0.00021018562454179737, P139=0.00017317730863692472, P140=0.0001349469463466907, P141=6.526479877197871e-05, P142=4.689440846938274e-05, P143=4.5807520464312724e-05, P144=2.8206133147236733e-05, P145=4.8556618363778694e-05, P146=6.0159251744704955e-05, P147=0.00010980691898287085, P148=0.00021951988852526676, P149=0.000555494625019452, P150=0.000680312809784877, P151=0.0003958573317090365, P152=0.00022839452679967065, P153=0.00013197545530647077, P154=0.00010049006965329589, P155=9.276947525078713e-05, P156=0.00010737173266458911, P157=0.00011767476847307898, P158=0.0003226059864445522, P159=0.00041982936448314504, P160=0.0004484176452857765, P161=0.0004017782411756461, P162=0.00048377004752048555, P163=0.0005749930634362864, P164=0.00042624302085035625, P165=0.0004264916633623256, P166=0.00038176607146271445, P167=0.0003689644245128738, P168=0.0003899481847161892, P169=0.00028096160620567684, P170=0.0002843905750002629, P171=0.00019963284193555273, P172=0.0002040690641126732, P173=0.00014321511296029447, P174=0.00013019727774815979, P175=9.899068451908448e-05, P176=8.706650690125584e-05, P177=4.6384215288807854e-05, P178=1.888697644050456e-05, P179=2.003432406162533e-06),Row( P0=-547.3179829211, P1=15.7700430813959, P2=-41.379858100632134, P3=-14.59974988323964, P4=-22.975533122122076, P5=-10.442683636363602, P6=-9.958205176979128, P7=-4.539510426906576, P8=-8.347821743674354, P9=-13.137231492936932, P10=-6.612778117837588, P11=-0.0752494962751301, P12=-1.0296360369255444, P13=7.746688730066633, P14=3.040585355842886, P15=6.252607096281402, P16=1.8391637468932813, P17=1.035943050221412, P18=3.758484688426677, P19=2.104139397601031, P20=-1.1616454825434746, P21=-2.3368292743563526, P22=-3.458113645069998, P23=1.7835112772072435, P24=1.6839227883934036, P25=0.46325740199836685, P26=-0.6860084964710579, P27=-1.4528248217482502, P28=-0.6022180968130878, P29=1.9434662054870133, P30=0.3526038090171504, P31=1.3087140125844596, P32=-0.03854263956290336, P33=2.326380302082871, P34=0.6203461257330183, P35=-0.16623349377516386, P36=-0.671960668710424, P37=0.5399118909091387, P38=0.9105282596942536, P39=1.8129427679679182, P40=0.6670074860333312, P41=0.653100310221827, P42=0.6031529296076882, P43=0.6011816930985313, P44=0.6288613136738931, P45=0.6124996111947396, P46=0.58096084367144, P47=0.566237845255327, P48=0.5548455222696099, P49=0.564676366112301, P50=0.5918458943348441, P51=0.5821100859489219, P52=3.546992018542886e-05, P53=1.7486465298956308e-05, P54=6.087206097673712e-06, P55=3.2965637341930916e-06, P56=3.21002543710047e-06, P57=2.5735256369893762e-06, P58=2.921938678124764e-06, P59=7.91573793803166e-06, P60=2.388480282668301e-05, P61=4.248535190777235e-05, P62=9.678440856609593e-05, P63=0.00013983960810834707, P64=0.0005533293262939676, P65=0.006855898667563348, P66=0.016845251153752298, P67=0.02540659268173649, P68=0.035893082153613205, P69=0.07040438233434734, P70=0.11413030973092493, P71=0.1971134132096222, P72=0.3536726308535476, P73=0.1999952417740087, P74=0.06772126840540088, P75=0.03260095980230254, P76=0.018211281906028838, P77=0.00962763773451816, P78=0.005171310434717527, P79=0.005446303193918639, P80=0.009174423640767424, P81=0.010418405496707134, P82=0.016285893069330922, P83=0.0664739958699338, P84=0.3058683933975932, P85=0.23530581754633142, P86=0.048972753537818464, P87=0.03440156071844989, P88=0.04113784742059513, P89=0.06102693435614452, P90=0.09261590619712147, P91=0.1313389040505165, P92=0.19606183990127066, P93=0.18391320611354972, P94=0.6239359784150581, P95=0.9111996001070585, P96=0.37581958417382755, P97=0.20510951442325204, P98=0.1347149023803989, P99=0.02077917941519976, P100=0.007168391826363464, P101=0.01011059141165386, P102=0.007436326366704075, P103=0.014297945552537394, P104=0.06912363117798226, P105=0.15169907126194715, P106=0.2856660212535531, P107=0.6605798691079584, P108=0.2150486324417355, P109=0.02137102172142745, P110=0.021062853055292093, P111=0.022545966435171455, P112=0.017269339741469998, P113=0.019875752540877475, P114=0.007396466888010787, P115=0.03687657030242606, P116=0.07505687534918346, P117=0.05617381911214383, P118=0.04922926013435455, P119=0.0356180759709182, P120=0.009548054553683601, P121=0.008637394038394005, P122=0.023280135040233668, P123=0.02439826923680498, P124=0.004922315470556582, P125=0.003414342436541104, P126=0.004043312465972061, P127=0.006161208091231248, P128=0.002785706343209412, P129=0.0017689702971904872, P130=0.002609471685414854, P131=0.0020084026025235643, P132=0.0005125302842126525, P133=0.0018544171448395715, P134=0.00305774425793483, P135=0.0010545728949545238, P136=0.0012858397270471477, P137=0.0015390482859758674, P138=0.0006813888175381764, P139=0.0005765529113479117, P140=0.00040013957507636027, P141=0.0006194607037281782, P142=0.0007793516049672306, P143=0.0003125927376550836, P144=0.00048409372794847375, P145=0.00039784132151906047, P146=0.00038244091750216285, P147=0.0007891554672312802, P148=0.0011441517174644671, P149=0.0009368206772751323, P150=0.0009818111651225714, P151=0.0008709328884994984, P152=0.0009743792111223396, P153=0.000753897277628998, P154=0.000547880959180431, P155=0.000566142230691272, P156=0.0005065943534025171, P157=0.00041468871442173523, P158=0.00040473306439609274, P159=0.0003118472116091878, P160=0.0005782591356181557, P161=0.0006564165332452645, P162=0.0007407790408958198, P163=0.0010558089777570427, P164=0.0010204003827277133, P165=0.001683964445082606, P166=0.0018009995279152603, P167=0.0018433623771764282, P168=0.0014503226948511068, P169=0.0017152233307626062, P170=0.0021985305902660988, P171=0.0023415345880159442, P172=0.001659092534407314, P173=0.0015887430759351044, P174=0.001288466118323506, P175=0.0011544062370256354, P176=0.0006617058131619798, P177=0.0005429593328288456, P178=0.0004372062966852016, P179=6.0673164177834535e-05),Row( P0=-507.80339542117014, P1=3.019637870148095, P2=-30.955855829990718, P3=-17.637976611598678, P4=-19.014187313051824, P5=-8.245759115416417, P6=2.467083727672947, P7=6.950726833158054, P8=12.638924398784452, P9=6.874793056879466, P10=-2.6704773082128437, P11=-7.012853160535042, P12=-11.581315093380844, P13=1.2121706485608228, P14=5.359579795469796, P15=5.96306637507453, P16=-1.4525331038686846, P17=-4.921444066419642, P18=2.901084852547757, P19=3.1630996113160212, P20=-2.6430591093496068, P21=-0.21083574829197183, P22=-1.508866645729228, P23=1.5091728959459205, P24=0.02002258663246033, P25=-3.031206204059765, P26=-0.34029419102203223, P27=0.10539436202572071, P28=-0.9503975359566439, P29=0.5438482603614286, P30=-0.4460053362763747, P31=1.5358817143713055, P32=-0.5008025665341712, P33=1.357814697273814, P34=-0.5511594048761878, P35=-1.1102346245362051, P36=-1.4232482225808665, P37=1.6389536953209216, P38=1.117543529349347, P39=0.15574054044085764, P40=0.745215440783364, P41=0.7435279805647226, P42=0.6818959604643591, P43=0.6232066625918264, P44=0.6132252558694605, P45=0.5188959808018477, P46=0.4979795784810445, P47=0.5355404931974324, P48=0.5742627437449616, P49=0.568296653467808, P50=0.5878452309098329, P51=0.6646191708481411, P52=1.6890775346664706e-05, P53=1.863380386683776e-05, P54=2.1661942961489347e-05, P55=3.1215820325096925e-05, P56=4.9376412527320534e-05, P57=6.104410641554035e-05, P58=4.140592627130013e-05, P59=3.7786696431755615e-05, P60=4.05621262324223e-05, P61=4.41634367737385e-05, P62=5.170354607322839e-05, P63=5.5911925657574944e-05, P64=4.403827268520016e-05, P65=6.465531373237071e-05, P66=0.00012059982043248074, P67=0.00012390421569778075, P68=9.141485026259284e-05, P69=0.00010275792164319062, P70=0.00013030185902154344, P71=0.00011253637321432856, P72=0.0001579988049334329, P73=0.0002149305423196046, P74=0.0005310460768531713, P75=0.000888193076282754, P76=0.0014552327140473955, P77=0.0052310722753917864, P78=0.011064545981259806, P79=0.01977676652939674, P80=0.04642392046569143, P81=0.07589048991990857, P82=0.13188965257762433, P83=0.3106283418703891, P84=0.7289528288273929, P85=0.8195615874668298, P86=0.9906994428134765, P87=1.7677428145409453, P88=1.2367630708022606, P89=0.90066102576953, P90=1.9820834236962057, P91=4.161884252146849, P92=1.1420975627191083, P93=0.03187694960494359, P94=0.0019205900001194145, P95=0.0009284132028367979, P96=0.0006450269032410868, P97=0.0005499999835739977, P98=0.0004744114467761127, P99=0.0005499206765176912, P100=0.003119668062881664, P101=0.06726076350082774, P102=0.2595069381202599, P103=0.19070411848296365, P104=0.17649011873076526, P105=0.44482247412718795, P106=0.5534911511767766, P107=0.6835524414967642, P108=0.7274571041189045, P109=0.33548348730255106, P110=0.015545396551972963, P111=0.0037936013350164666, P112=0.007126737788527116, P113=0.005507957121307207, P114=0.0023966618830061973, P115=0.002217320788224081, P116=0.0015547139867714062, P117=0.00493893881358519, P118=0.04254032908513267, P119=0.12170703662768273, P120=0.04838692636121276, P121=0.03746075090617801, P122=0.01085195857825337, P123=0.0067040284114556585, P124=0.004903617999958218, P125=0.00899144916060895, P126=0.002936278656357932, P127=0.008581225650184588, P128=0.013989936294852661, P129=0.015397228309603765, P130=0.014026583619059925, P131=0.006080000783769166, P132=0.05089287550037256, P133=0.018953932782656384, P134=0.015259099855791304, P135=0.005388696033485464, P136=0.011488491992799494, P137=0.008975322288757855, P138=0.004117805956686932, P139=0.010812773136618065, P140=0.007444199816962673, P141=0.006726846078007811, P142=0.0007590242397909029, P143=0.0004937029067547212, P144=0.0025202997512557136, P145=0.0030487397772521103, P146=0.0029319339819561152, P147=0.002837592755959369, P148=0.0029127737772121675, P149=0.009822892599015755, P150=0.005009766653819776, P151=0.0032895996045758285, P152=0.0010868154345091028, P153=0.0008496684547779062, P154=0.0006754655000094326, P155=0.000348834609371883, P156=0.00034814035100058443, P157=0.00046008775372632113, P158=0.0004604579185726007, P159=0.0006180804202779506, P160=0.00046177209789419557, P161=0.0006352626537909291, P162=0.0004895233678270238, P163=0.0007088378172977948, P164=0.0007682504465152424, P165=0.0006843637261314619, P166=0.0009639375242344752, P167=0.0007875728637656107, P168=0.0008501619869568344, P169=0.0008416893976094983, P170=0.0008161518736285781, P171=0.0009785960125541305, P172=0.0008993306554032025, P173=0.0006881643096270866, P174=0.0004867529900062891, P175=0.0004498665376075571, P176=0.00042798897009876203, P177=0.00047024240648736276, P178=0.00043128731025338237, P179=4.187945711827027e-05),Row( P0=-724.582335246358, P1=41.828936429808216, P2=-17.20723983892325, P3=-6.376392845515074, P4=-18.727417886629024, P5=-13.33278250068483, P6=-12.117171195976281, P7=-6.890400273907016, P8=-7.3977825785828975, P9=-6.925210708646191, P10=-7.74359856550841, P11=-5.5256959007442354, P12=-1.39984025465742, P13=-0.5382611801977397, P14=-0.4244495865616917, P15=6.426622847686161, P16=4.920016420981425, P17=4.882045595369347, P18=8.633719487345543, P19=5.867059386588436, P20=1.7527487279688991, P21=-2.203845317678272, P22=-5.702780626257746, P23=-7.910216573909273, P24=-5.1509136836495175, P25=-3.590147945731768, P26=-2.420718746996806, P27=-3.964071589187013, P28=-4.894111611511017, P29=-2.8656986580041415, P30=-2.171757854638516, P31=1.6373116663015812, P32=-2.839034066821291, P33=0.6209560464667276, P34=-0.10995022239023046, P35=2.4683095911893145, P36=1.0938084675025346, P37=2.232238947665512, P38=-0.5108136225875188, P39=1.651183386679376, P40=0.5064476352599419, P41=0.45275636506789463, P42=0.45302412951110005, P43=0.4537079780417654, P44=0.476011392209338, P45=0.4368127279941509, P46=0.4249651032423287, P47=0.39997839611776453, P48=0.44054940931344694, P49=0.5272561672614651, P50=0.5429519301022356, P51=0.5740583400702107, P52=5.712842146086118e-07, P53=3.98269809199768e-07, P54=7.50668728907476e-07, P55=6.362910453512434e-06, P56=2.3326597779256447e-05, P57=3.6779873145019396e-05, P58=1.7402622549081558e-05, P59=4.602175489994893e-06, P60=1.8350005497198122e-06, P61=3.8734512260850185e-06, P62=8.105843823041609e-06, P63=4.698728080828059e-05, P64=0.0013121707477579733, P65=0.006522746248536299, P66=0.01840132846046271, P67=0.02182110233037009, P68=0.00580024979762337, P69=0.005484086392803558, P70=0.011034388473486713, P71=0.010210839841859744, P72=0.0031838378356285756, P73=0.00060610681375653, P74=0.00022809422994104488, P75=0.00018987102950976907, P76=0.00013070122755036166, P77=0.0003609361473414472, P78=0.001500039914655476, P79=0.002365014750754641, P80=0.0015558152771047175, P81=0.003120448839044623, P82=0.008176826797400942, P83=0.004413780209467676, P84=0.0009688701822051477, P85=0.0005043617304944733, P86=0.00017714922397226403, P87=0.0007754394283759812, P88=0.005647095652581543, P89=0.007153862106131157, P90=0.015180937187669646, P91=0.01153565243426341, P92=0.0026963890816663785, P93=0.0009825059255514756, P94=0.0037873946124680914, P95=0.0013978885591232901, P96=0.00012265891354864362, P97=2.6786024286467386e-05, P98=2.3587776171761052e-05, P99=0.00011062424093046544, P100=0.0005390826222103215, P101=0.00032353020369993804, P102=0.000738413289416788, P103=0.0016650384572495147, P104=0.0002240390861726782, P105=6.238915211126736e-05, P106=0.00025904651369747917, P107=0.0006215048503785614, P108=0.00032325518959118975, P109=0.0008098285425161372, P110=0.0010113121614758168, P111=8.918617115150338e-05, P112=0.0005253665358898502, P113=0.0005016212610332203, P114=0.00022860999442416897, P115=0.0009404443667855307, P116=0.0005156776171898346, P117=0.0003794458029737782, P118=0.00027010992257096617, P119=0.00010757072457213242, P120=0.00013992211941498214, P121=0.00018361354497691638, P122=8.577925538984646e-05, P123=3.915945703444493e-05, P124=7.317773247877847e-05, P125=4.992109621512176e-05, P126=1.2594577213915987e-05, P127=1.4664196365276482e-05, P128=2.7323934133012966e-05, P129=1.6532357105650152e-05, P130=6.4267787909403975e-06, P131=3.758017083070606e-05, P132=2.097532933670146e-05, P133=2.7119460811504896e-05, P134=3.591062795211453e-05, P135=2.7999641738845834e-05, P136=5.0475142590380543e-05, P137=2.898434881681841e-05, P138=1.6055754483889306e-05, P139=1.6078477373030246e-05, P140=2.885718813320828e-05, P141=1.9458302426367714e-05, P142=2.0653727592176474e-05, P143=1.3623727666496067e-05, P144=9.707076369802404e-06, P145=9.686240728423857e-06, P146=6.330596417838196e-06, P147=6.1737397968174205e-06, P148=6.209223121361398e-06, P149=9.8664183965511e-06, P150=2.0683701135644963e-05, P151=9.644040214341227e-06, P152=1.1076839751214526e-05, P153=8.010273868719723e-06, P154=6.814378616026622e-06, P155=5.873431975414253e-06, P156=6.715063826464791e-06, P157=9.132836432117545e-06, P158=8.983216291892684e-06, P159=1.1328479449020491e-05, P160=2.1121205434407394e-05, P161=1.665284470584016e-05, P162=2.152464843973632e-05, P163=1.8260717034862764e-05, P164=1.72863887823614e-05, P165=2.0326697762822512e-05, P166=2.249777255146568e-05, P167=1.7767971213472524e-05, P168=2.0928236481462962e-05, P169=1.690341819556945e-05, P170=1.926951234281559e-05, P171=2.2375695541202072e-05, P172=1.4532341455634444e-05, P173=2.1841615070923033e-05, P174=2.78953608179536e-05, P175=2.138124994518702e-05, P176=1.947849629230781e-05, P177=2.4081354630820734e-05, P178=1.2825665817325265e-05, P179=1.2599222340929925e-06),Row( P0=-563.6557323782409, P1=24.065253231229402, P2=-28.427709287376782, P3=2.1363651817734515, P4=-20.38245722644982, P5=-5.936740900473123, P6=-0.08109203856621536, P7=-3.0305260956383187, P8=0.24269085085960956, P9=-3.6821318228463444, P10=-3.37899652963726, P11=-0.3361412722993818, P12=-4.031018861796249, P13=9.237015534159292, P14=1.5875016278669003, P15=6.781201839523238, P16=8.576247824187618, P17=4.503232415338583, P18=10.389118557962409, P19=4.849642880236435, P20=2.79114896754985, P21=0.2443389504956988, P22=-4.2807982779300175, P23=-1.9679884855038967, P24=-4.083725498875122, P25=-1.8772462454588275, P26=-0.680548043290013, P27=-1.5597804155709645, P28=-3.0099726706439363, P29=-0.5851128764465275, P30=-1.636198949171508, P31=2.346957134064785, P32=0.08490327908820558, P33=0.7827961686965322, P34=-4.815412747928796, P35=-0.3130929648290447, P36=3.4000155713374896, P37=4.062717290585028, P38=-0.07650402874892744, P39=-2.6915938948402505, P40=0.4892800316315589, P41=0.5278306833492858, P42=0.5609589088407274, P43=0.46482274285490294, P44=0.4421084091823952, P45=0.5228291228072512, P46=0.48268757890316555, P47=0.43789411563714226, P48=0.4843504585492947, P49=0.5182443062806056, P50=0.5626545578108941, P51=0.4914913567454181, P52=0.0033188023629053924, P53=0.0034773512273427567, P54=0.0013905974135101373, P55=0.00039789872809427204, P56=0.00021770301481774049, P57=0.00011613816456263378, P58=4.721465717930762e-05, P59=2.5344689364405136e-05, P60=1.860598844204735e-05, P61=2.4988420122952647e-05, P62=6.756967195352355e-05, P63=0.0003557391292307809, P64=0.0035202335190462034, P65=0.030245981988837118, P66=0.059349765401171095, P67=0.05766619642152837, P68=0.05050957341182222, P69=0.07664395104755456, P70=0.049267021712505826, P71=0.020644480326973018, P72=0.034505204821356154, P73=0.07749039955680367, P74=0.10479291844810636, P75=0.0775538393538442, P76=0.026172378021055955, P77=0.007857379659226487, P78=0.0052095664471830784, P79=0.011686688006861755, P80=0.04187207876770495, P81=0.04245839315188869, P82=0.03366221433775674, P83=0.08016772913172505, P84=0.11532873866813995, P85=0.07781584822014435, P86=0.16796121713248394, P87=0.27557780497602696, P88=0.1731837585463164, P89=0.08646101654952457, P90=0.08185517560657542, P91=0.061954988914896036, P92=0.16328840837063918, P93=0.25176545120143917, P94=0.11800824425565304, P95=0.024010372100578322, P96=0.007736291045907598, P97=0.03821325849839277, P98=0.07222012010272516, P99=0.10370026365977522, P100=0.06692218895770798, P101=0.04938189917594796, P102=0.1040573538335649, P103=0.022517105336727607, P104=0.003637149327974694, P105=0.03587848730926982, P106=0.16600056715948183, P107=0.5153799948394222, P108=0.43325631301376305, P109=0.083880967845129, P110=0.012431880928029439, P111=0.0032794901703892074, P112=0.012652953583729688, P113=0.05215923223867283, P114=0.11743927552160639, P115=0.04546870976218694, P116=0.004682929873775839, P117=0.0027697429977933006, P118=0.014279250635623478, P119=0.058984946240461446, P120=0.01215170723099854, P121=0.013440858741956717, P122=0.014414162821468745, P123=0.01546246613857877, P124=0.010076545781890973, P125=0.005194941409383231, P126=0.004989249162565355, P127=0.01322811266452382, P128=0.009136328844061776, P129=0.016860355447907364, P130=0.018408035419499955, P131=0.014133532741788486, P132=0.004681196198847181, P133=0.0035317627428812053, P134=0.00262032868993818, P135=0.003466261583919826, P136=0.006586641587126456, P137=0.0013079010724581983, P138=0.0035178373140415313, P139=0.007254269801818454, P140=0.001956178133088631, P141=0.0011775215882436295, P142=0.004976225341446621, P143=0.002234303501373186, P144=0.0011498246165978054, P145=0.0015672273931471388, P146=0.0020518875588525034, P147=0.0021681846186292634, P148=0.004912516517202996, P149=0.005214232156701537, P150=0.006804822883668365, P151=0.005110990906397281, P152=0.0035139121839180636, P153=0.0015267667515893569, P154=0.0003650274707559312, P155=0.00021065457144008487, P156=0.000254964764594504, P157=0.00012991291255172587, P158=8.741959563547894e-05, P159=0.00011074032067660064, P160=0.00010213464234078524, P161=0.00010592199021782114, P162=0.00013088803751634546, P163=0.00016857677121538396, P164=0.00024651359976828106, P165=0.00018693926692423739, P166=0.00024099632084909265, P167=0.00018597899160805102, P168=0.0001489608748539271, P169=0.00023183617470764998, P170=0.00014629483514541148, P171=0.00013142646563885833, P172=0.00010306380549335601, P173=0.00011003262186188014, P174=0.00010808048671137186, P175=5.045878957937473e-05, P176=3.3428463810720664e-05, P177=2.8158138753445027e-05, P178=2.008292463374167e-05, P179=3.682914368158796e-06),Row( P0=-574.7102583070886, P1=15.763129658158391, P2=-34.265391909173886, P3=-9.999679145534241, P4=-25.62033243531293, P5=-13.621907272925297, P6=-7.085200144702972, P7=-6.9756752015966335, P8=-6.905879659628224, P9=-6.581996399676823, P10=-6.756210115782623, P11=3.8421487078034886, P12=3.256845057928464, P13=14.582206455401636, P14=6.6691170912627005, P15=12.291652061919045, P16=2.0579111247929744, P17=-3.6120187796165837, P18=0.39897879053494356, P19=-5.161583164667963, P20=-1.0612747128913953, P21=-1.7138963545508425, P22=-5.5950153071033535, P23=-2.0282641836937803, P24=-3.722701673221872, P25=-0.4862792139994099, P26=-0.8117286640935926, P27=0.9512644381582966, P28=-1.0667325191089434, P29=-0.16981756763548242, P30=1.0263656729709387, P31=1.5414634308462547, P32=-0.38603561941841663, P33=3.1447664530645008, P34=-2.6878767078735124, P35=0.7179291868845386, P36=-0.9342780708778091, P37=1.0823069462586488, P38=-1.190371963719759, P39=-1.6965659728281797, P40=0.5318630046158002, P41=0.5477829387647909, P42=0.5893149542861765, P43=0.5692016004474821, P44=0.5623915598119275, P45=0.5395214413117729, P46=0.5158228209879987, P47=0.4724103070590512, P48=0.47315890015993356, P49=0.5268664892991047, P50=0.5200369849461564, P51=0.5119465147319294, P52=2.3437892237108244e-06, P53=7.0541847370046665e-06, P54=1.409351370779794e-05, P55=1.9317062846657936e-05, P56=1.7190670475451786e-05, P57=2.0757474952465203e-05, P58=1.2139998634292312e-05, P59=5.2752824349187586e-06, P60=5.659654323797717e-06, P61=8.742458637770596e-06, P62=1.2260637585943398e-05, P63=1.235790940653762e-05, P64=4.40150540933943e-05, P65=0.004221871175151003, P66=0.023909021497890218, P67=0.031340836318546506, P68=0.05806596228469061, P69=0.1283804207794921, P70=0.13458276543549139, P71=0.2317925047036051, P72=0.18334347123454806, P73=0.0911079849495528, P74=0.0933505035914191, P75=0.1008975925147752, P76=0.02652417032553171, P77=0.0027901789704584054, P78=0.001442804602318155, P79=0.00121977862974561, P80=0.004182377785181465, P81=0.010548717608731948, P82=0.01209742020765266, P83=0.012652627730727914, P84=0.040370459526333946, P85=0.11815661520644151, P86=0.12824782837116358, P87=0.1193400475831172, P88=0.17085630634660023, P89=0.1184465440703718, P90=0.12082545417725546, P91=0.2388591352648816, P92=0.19590756248173447, P93=0.14100165715620275, P94=0.16556400275223773, P95=0.14571980645403945, P96=0.06529063639008695, P97=0.09662082445708653, P98=0.0747207512356312, P99=0.113378050200249, P100=0.3002487113632057, P101=0.11685957026228752, P102=0.06240702247072992, P103=0.06879151764929534, P104=0.07871730626695192, P105=0.09233653216775214, P106=0.04590952468108693, P107=0.018967892958100398, P108=0.07829211268955966, P109=0.05844436110331739, P110=0.01840952746540478, P111=0.018104603763366797, P112=0.04199596974000733, P113=0.03573154226758227, P114=0.02592063092039871, P115=0.056399773873118356, P116=0.02808823679558136, P117=0.0318776870056745, P118=0.010959664935521774, P119=0.003711010047264935, P120=0.034091961812125204, P121=0.06466535926572392, P122=0.01105541825057014, P123=0.0021080691783570793, P124=0.007588950364576128, P125=0.009761090272946384, P126=0.018061517414557127, P127=0.004029043798528447, P128=0.0038172305004925975, P129=0.0027174959388626178, P130=0.0039025978874668026, P131=0.004778981930514523, P132=0.0007538099599753739, P133=0.001424145303402631, P134=0.002116104954178689, P135=0.001294518948479928, P136=0.002199847329437233, P137=0.002820845060086372, P138=0.0013819614571061241, P139=0.0010201246925627787, P140=0.0011998227018358342, P141=0.0005283949754841006, P142=0.0004176655435644292, P143=0.000570497872252638, P144=0.0002622037927281131, P145=0.00033791068037642325, P146=0.0005221034301180925, P147=0.0007550738738123091, P148=0.0008171146627560272, P149=0.0005963538019681956, P150=0.0007971908433737941, P151=0.0017674398252962905, P152=0.0016488264162140642, P153=0.0011060748601021276, P154=0.0005243461976643778, P155=0.00036542286674385866, P156=0.000302992119431421, P157=0.0005135609272739546, P158=0.0013431870829665237, P159=0.0019223975422455028, P160=0.0011632112131058938, P161=0.0010861132725362521, P162=0.0015891617837824835, P163=0.001396891838818803, P164=0.0014982303316715964, P165=0.0012782596578114038, P166=0.0011344767530661964, P167=0.0013163805226267824, P168=0.001162923661268087, P169=0.0010577790329832576, P170=0.000747595407525177, P171=0.0004934652390713654, P172=0.0006787855400563354, P173=0.0006597257710019915, P174=0.00046269120051207476, P175=0.00022499402243969302, P176=0.00013714631769365444, P177=0.00010028244556546899, P178=4.934217186306832e-05, P179=4.872317764610045e-06),Row( P0=-617.2518348324922, P1=37.16250381162352, P2=-28.505774304459873, P3=-7.079697671450459, P4=-16.91785321943138, P5=-14.904383540399774, P6=-8.799978564736662, P7=-5.0134817313173015, P8=-7.223646122728138, P9=-5.22713110581947, P10=-10.32680523622813, P11=-3.3911413718043106, P12=-7.100433169837882, P13=-7.553122489663204, P14=0.4396845278031892, P15=-4.30294159262258, P16=2.8932715749890603, P17=-2.1709210399821317, P18=2.775253140012809, P19=4.4171228760811205, P20=0.6428325869918289, P21=9.456536898451544, P22=7.656577040136964, P23=9.321725769540121, P24=6.770133544847224, P25=3.256498916413341, P26=-1.4318681670111388, P27=-1.8110120098080418, P28=-2.0265633892994552, P29=1.1626352927020671, P30=-0.5309531174468999, P31=0.39898716813906693, P32=-4.210262452288572, P33=-2.0242206542999535, P34=-3.804199839518012, P35=0.5223523978477516, P36=-0.8764060088586163, P37=-0.3837233448981193, P38=-1.800876688393123, P39=-2.5268221752545132, P40=0.6689579248185412, P41=0.6772580785867073, P42=0.6478160536995057, P43=0.6113027717107264, P44=0.616463206269996, P45=0.650328457229042, P46=0.7388188186337226, P47=0.7196857445742958, P48=0.624936762863542, P49=0.6454917410270518, P50=0.6636075314023694, P51=0.6428007348563499, P52=4.912333202097128e-07, P53=2.00757374469704e-06, P54=3.2451599590116386e-06, P55=2.074028607961434e-06, P56=1.8198082708907147e-06, P57=2.1404980131766495e-06, P58=2.025218686590577e-06, P59=1.5842446957136556e-05, P60=0.0002383948234520805, P61=0.0019797923709720887, P62=0.016547022463249156, P63=0.031585681500991065, P64=0.015408853455837443, P65=0.015097988673473358, P66=0.024775471812758344, P67=0.02509950607229879, P68=0.016059047512381654, P69=0.006661537797240109, P70=0.00220050054808565, P71=0.0009098607050300519, P72=0.001709117211496793, P73=0.005700205700305042, P74=0.01643543432921124, P75=0.024413346160230665, P76=0.012801645828905556, P77=0.015509994853701692, P78=0.029569973077427897, P79=0.027540418351698175, P80=0.0177068339141053, P81=0.015884725599758558, P82=0.02819711836503484, P83=0.07097496667506456, P84=0.10072884853555737, P85=0.17094633179766336, P86=0.1451254396123895, P87=0.07983811290411122, P88=0.02458648447846722, P89=0.004610897360420311, P90=0.003152760822914094, P91=0.0030546123854205973, P92=0.01006279249583948, P93=0.01571991284899778, P94=0.02814836705625386, P95=0.04014444970357846, P96=0.011530751590709903, P97=0.011593872583579455, P98=0.009602072616718527, P99=0.008467042189967421, P100=0.004926522501619318, P101=0.005502836490737563, P102=0.012227416349844516, P103=0.008071273849396746, P104=0.009652450050095617, P105=0.006575760187857481, P106=0.005703288862779423, P107=0.006620676926634432, P108=0.003089362360484116, P109=0.0033096676098918845, P110=0.0065844443545324234, P111=0.007509876722438785, P112=0.003031482394400335, P113=0.000775447854411914, P114=0.0022788200880482458, P115=0.002858596430021805, P116=0.002562649373471262, P117=0.002365859338176789, P118=0.0014479698122330646, P119=0.0008160596019070536, P120=0.0025087887057952908, P121=0.0015153432402414233, P122=0.0007953631067412969, P123=0.0014928691627284516, P124=0.0020776823494208256, P125=0.0006120329903241273, P126=0.0004722350054893954, P127=0.0018172429202555757, P128=0.003921736259242339, P129=0.0008019775866652203, P130=0.002737389647497619, P131=0.0033225132599739547, P132=0.0035186390852426615, P133=0.002819418171315959, P134=0.002274938885896245, P135=0.00195762002447304, P136=0.001664682969496634, P137=0.0013684020797056838, P138=0.0028196484358883676, P139=0.0016020398154073215, P140=0.0011045094335130896, P141=0.0005594871724340051, P142=0.0004013207249908878, P143=0.0005488115413873975, P144=0.0005152990521959234, P145=0.0008801783476386525, P146=0.0008868856803640019, P147=0.0005056281282619266, P148=0.00016039271302011136, P149=8.1290470430343e-05, P150=0.0001176431813689441, P151=9.850182244898245e-05, P152=5.875864397506005e-05, P153=5.633171092841892e-05, P154=3.4183215551591876e-05, P155=2.543785294600346e-05, P156=1.5430370450172477e-05, P157=2.8590843072370673e-05, P158=4.255361781817381e-05, P159=6.893499536460048e-05, P160=6.923264597068275e-05, P161=7.005884183357462e-05, P162=7.155847834409866e-05, P163=6.054695595104126e-05, P164=5.78674571021737e-05, P165=8.005992489842502e-05, P166=5.85262294289612e-05, P167=5.3244142335344846e-05, P168=3.4003495883395964e-05, P169=2.865455397984563e-05, P170=2.155683126764417e-05, P171=1.0830747666251413e-05, P172=1.6881215015825575e-05, P173=2.5557730614492943e-05, P174=1.9548694602855536e-05, P175=1.5736567987207492e-05, P176=1.4712043233435468e-05, P177=1.1738555097200217e-05, P178=5.188632131364987e-06, P179=6.38194179544558e-07),Row( P0=-528.7389052276183, P1=19.9609147326513, P2=-21.79358032178349, P3=-18.267428597558606, P4=-17.274781309436534, P5=-23.134818137835598, P6=-17.2284891250635, P7=-9.663186870058002, P8=-10.198898078167444, P9=-7.171666629899881, P10=-8.107404540332887, P11=9.584092667597346, P12=3.5892959545686507, P13=4.010447851252101, P14=6.181233440429043, P15=-2.6722911160052205, P16=-3.2519482926122936, P17=-3.0200729351239395, P18=2.831272182767702, P19=0.7039794713313745, P20=1.246099890543361, P21=3.104775948857947, P22=1.1114667462831456, P23=2.634939818391583, P24=-1.524702521251954, P25=-2.6062116183673196, P26=-4.010263828902359, P27=1.3694950000821784, P28=-2.8974813800786303, P29=0.5406198960226936, P30=-1.4795670232153824, P31=-0.6924951762361359, P32=-1.9375482085785432, P33=-0.16344654887230523, P34=-1.610141274748056, P35=3.5914875050982547, P36=-3.5094236425160426, P37=-2.9676662801528133, P38=-0.6645081149006954, P39=1.6748343645362511, P40=0.5928104792493651, P41=0.5814103560634568, P42=0.607521215348316, P43=0.6792413013259573, P44=0.6714770843779723, P45=0.6378716938407215, P46=0.686029422503539, P47=0.7442158186910152, P48=0.7137987477238038, P49=0.6513962626368667, P50=0.6218257941501564, P51=0.5998990746881748, P52=1.8070237794334672e-06, P53=7.06429003973689e-06, P54=2.116537644735183e-05, P55=4.346939136635702e-05, P56=6.154652206217255e-05, P57=8.853610640219894e-05, P58=8.712370228292185e-05, P59=5.945383586633047e-05, P60=5.258400176298492e-05, P61=0.00018362480878884003, P62=0.0012917017434533424, P63=0.006438306105620465, P64=0.017716359702331562, P65=0.032530869693870056, P66=0.05549398117917623, P67=0.08382655383605948, P68=0.10428182063289139, P69=0.16169545044398612, P70=0.31016790830771124, P71=0.4981953196471899, P72=0.33629988108356285, P73=0.2425084092456169, P74=0.6204464406825306, P75=1.3744842242946005, P76=1.2370765368942827, P77=0.8731380412173462, P78=0.47494216246212506, P79=0.13844816028343318, P80=0.01524642151090975, P81=0.003865730934775712, P82=0.0028277495331756057, P83=0.004752494096643158, P84=0.006760460557264317, P85=0.007417945876555142, P86=0.011969613864497837, P87=0.028383047164864325, P88=0.07923862114481706, P89=0.1309390083964047, P90=0.30079077954943345, P91=0.2540443060168477, P92=0.12096567127972807, P93=0.057232889921871595, P94=0.09871896037089026, P95=0.16086648009748228, P96=0.21728643275073173, P97=0.31091634098475884, P98=0.13786633752610775, P99=0.015054023894958461, P100=0.004498198668896868, P101=0.0071999421223344386, P102=0.07245139745230184, P103=0.089552618979863, P104=0.019256669146361392, P105=0.011714458183846258, P106=0.01571247688171999, P107=0.019894071895790684, P108=0.02276314179659848, P109=0.060629846642900574, P110=0.20537976530266433, P111=0.09754181443568721, P112=0.030252997807001726, P113=0.008426992458899902, P114=0.00611870649181526, P115=0.009047086626477664, P116=0.01357492919922674, P117=0.02233476805091205, P118=0.06495418324213419, P119=0.054346640567558745, P120=0.025592469824436467, P121=0.019521396013585383, P122=0.03129655682826879, P123=0.02986237637212971, P124=0.0332101156839291, P125=0.06400306371307624, P126=0.018876202730418096, P127=0.011550145230960174, P128=0.007478580510672395, P129=0.010505112072764056, P130=0.0045029709648221035, P131=0.0010122938642137056, P132=0.004046718960701192, P133=0.0047941027690819775, P134=0.003895747890245616, P135=0.0013793531176693366, P136=0.001036056776169641, P137=0.001976760520923566, P138=0.004479961767245491, P139=0.003983414165648694, P140=0.00514661718789757, P141=0.002212388042258346, P142=0.0007991267830755011, P143=0.0005637626748379281, P144=0.0005399104830664019, P145=0.001412681796498167, P146=0.000937473124835564, P147=0.0006473029829151232, P148=0.0008151886012358829, P149=0.0006740581688365871, P150=0.0006721633783047148, P151=0.00037406750693634816, P152=0.00033748096254120253, P153=0.0004988671221664422, P154=0.0009363111515450343, P155=0.0012220065143546414, P156=0.0013769745388839412, P157=0.0011305784271695668, P158=0.0010919056800002495, P159=0.0017130919050658945, P160=0.0015403411984339643, P161=0.0011912412829716233, P162=0.001284365218573808, P163=0.0022587115634770017, P164=0.0020880405615457866, P165=0.0017878410576162409, P166=0.001795785525578767, P167=0.0012313420998562984, P168=0.0008543971816206823, P169=0.0004635096422157656, P170=0.0007981516469780834, P171=0.0009554958245608945, P172=0.0009032769900465591, P173=0.0005843845557825363, P174=0.0006884579862257034, P175=0.0008251186294463032, P176=0.0013131960493896015, P177=0.0014557923743129244, P178=0.0009097358363218062, P179=0.00011555960508520611),Row( P0=-593.1074674908642, P1=34.844455586653694, P2=-25.10473752056909, P3=-11.489529517084796, P4=-17.70332962139993, P5=-12.454162865790117, P6=-9.68203591285626, P7=-5.04919831623828, P8=-11.669673644649134, P9=-4.971997587471232, P10=-11.64434625411899, P11=-5.622194723608732, P12=-4.788857991113422, P13=-11.648500781381992, P14=0.21191101089276224, P15=-2.7163364921594666, P16=1.3594564608279227, P17=0.7306434005732405, P18=3.284248167500065, P19=4.6991287222929055, P20=2.4332026101493063, P21=8.17695078788714, P22=4.560439890227321, P23=6.077252729198894, P24=4.359961292774848, P25=1.522195883075762, P26=-1.8271147802740446, P27=-0.6514172338321924, P28=-2.356372826469835, P29=1.0216529124106961, P30=-0.8424355116424591, P31=0.3931522789695844, P32=-4.082572092256909, P33=-1.1593124903771586, P34=-1.5008226767268353, P35=0.3849314860619943, P36=-0.9495389146264854, P37=-0.6125406939547045, P38=-2.6695531574612925, P39=-0.11754192439170118, P40=0.6326821884566478, P41=0.6140392809230744, P42=0.6297250041035465, P43=0.6063628696098227, P44=0.6142990273709554, P45=0.6523017300385224, P46=0.670808460216597, P47=0.640014870682877, P48=0.6242935555346112, P49=0.6336780419772331, P50=0.6419361513802728, P51=0.6469974013268427, P52=4.408933547188141e-07, P53=1.7134918127101612e-06, P54=2.712197181123386e-06, P55=3.386178432826625e-06, P56=1.1785002782162682e-05, P57=1.704551301025489e-05, P58=6.390093452407375e-06, P59=3.4894717671737774e-05, P60=0.000431775747550209, P61=0.002787504163201139, P62=0.017027482975668477, P63=0.03754506838940225, P64=0.033854987545380945, P65=0.031091598966082993, P66=0.04983097439180069, P67=0.049119096090070605, P68=0.02583973429762206, P69=0.014650475207881552, P70=0.006541855787670434, P71=0.0016896344535717465, P72=0.0011341113279927843, P73=0.00377880143878153, P74=0.007905925279134043, P75=0.01756566143616781, P76=0.01991121001892292, P77=0.042171743429697824, P78=0.03755137586603464, P79=0.021248017267148495, P80=0.03152367959696878, P81=0.03392622958074409, P82=0.03206759540505544, P83=0.04502342861284742, P84=0.0653278598356851, P85=0.07352572663193824, P86=0.16515047672156366, P87=0.29964238398388043, P88=0.15635475842759752, P89=0.08335871624128452, P90=0.07564872257895186, P91=0.010441286103617674, P92=0.022568295880749614, P93=0.04458642200836728, P94=0.14131019627161093, P95=0.08787014620716398, P96=0.04101252227828755, P97=0.01984313109829076, P98=0.025585599347694832, P99=0.022242111849319554, P100=0.017016895407669517, P101=0.03375608215390943, P102=0.08004814796200398, P103=0.22732362854550708, P104=0.10410710624260652, P105=0.02783397255478468, P106=0.006072084487014264, P107=0.006880809517436394, P108=0.004196869041557431, P109=0.010971925835992024, P110=0.03426629363600717, P111=0.02008346571171486, P112=0.006641034281931527, P113=0.0016162120103371653, P114=0.002860635251364349, P115=0.004422913125209385, P116=0.0033582049746594153, P117=0.00583050646768903, P118=0.003635931318157996, P119=0.0037911725526147255, P120=0.002626559454634604, P121=0.004305655836542091, P122=0.002708932854068729, P123=0.0015976855933954638, P124=0.0015551718071237415, P125=0.009946663672850872, P126=0.004700909361160636, P127=0.0007211476595431198, P128=0.002566565324464086, P129=0.0038665651022758455, P130=0.0018385054128178622, P131=0.0006149954042201731, P132=0.001556000413657845, P133=0.001678342290024357, P134=0.0006485229599050283, P135=0.0015214163866456968, P136=0.004180292120225904, P137=0.0018599637879783137, P138=0.0011685293891664878, P139=0.0008017733555219038, P140=0.00044499010726105944, P141=0.000631556060789861, P142=0.0005851947699463554, P143=0.0007470370480018828, P144=0.0006904792635313884, P145=0.0009984144122876246, P146=0.0011120580156936102, P147=0.0011739217439906897, P148=0.000692984372951244, P149=0.0006640627671512488, P150=0.0002991607822727618, P151=0.00019996904171973865, P152=0.00013928192484697095, P153=0.00011549042916008748, P154=0.00012484187803676734, P155=0.0001648651344689114, P156=0.00010577966146554833, P157=0.00012166795176077863, P158=0.00014860397355342277, P159=0.00021823740489638249, P160=0.00029688497314915607, P161=0.00022090733796599485, P162=0.00024007339681864846, P163=0.00019178147846101319, P164=0.00013865158375031425, P165=0.00010832376817891764, P166=8.948903670315597e-05, P167=7.917159273683787e-05, P168=6.739655969291201e-05, P169=5.9821585915133145e-05, P170=6.524993670969428e-05, P171=4.876125727814687e-05, P172=2.774424754525646e-05, P173=3.6857855368031135e-05, P174=3.304807471341734e-05, P175=3.435751311164105e-05, P176=5.651034285744903e-05, P177=4.4914703689549106e-05, P178=1.9435074303988776e-05, P179=1.7880157285702134e-06),Row( P0=-599.4610763299535, P1=25.156929620417568, P2=-32.99617808860402, P3=-7.851066243256777, P4=-22.231317526532116, P5=-20.742500830529785, P6=-10.227038651494833, P7=-3.7157891807905297, P8=-1.8137660830135234, P9=-10.587507904003994, P10=-11.23217170126569, P11=-3.076189654018018, P12=-10.8348437685873, P13=-6.763721471874835, P14=0.35184537495399365, P15=2.685998865115452, P16=8.610863177151938, P17=13.213751221154237, P18=19.28568894122734, P19=11.843894065896151, P20=6.3506668537313535, P21=2.430032045680435, P22=-4.7168368366648625, P23=-3.776844426627326, P24=-2.039573613063936, P25=-0.17611859826940812, P26=-3.078702965790481, P27=-1.8576329089379142, P28=-7.369194651188248, P29=-1.479356003025925, P30=-2.964599317521677, P31=1.0455503750353807, P32=-3.231257363922646, P33=1.4054164249367838, P34=-1.501423717432174, P35=3.8926858571986864, P36=0.3810891990284835, P37=2.793813287593022, P38=0.6516444647295068, P39=-0.9370178039111785, P40=0.6264127743925625, P41=0.5228998805665775, P42=0.5076475386759003, P43=0.5426191177463537, P44=0.5664609413380377, P45=0.5514921334716407, P46=0.5567861404875395, P47=0.5818715980644703, P48=0.669686668927267, P49=0.7362016399362575, P50=0.7694434063885888, P51=0.735974566808379, P52=4.5486405646820575e-07, P53=2.5093585537372032e-06, P54=4.0896573526004476e-06, P55=1.8408077203785575e-06, P56=2.2679997879759186e-06, P57=3.0117579927431636e-06, P58=1.5860143971764956e-06, P59=1.8522499786611512e-06, P60=5.9839719838982305e-06, P61=5.5231667132698206e-05, P62=0.0018034947976327863, P63=0.007813462924638297, P64=0.016692416949890144, P65=0.07918096179450541, P66=0.1477859077218503, P67=0.09367122914725273, P68=0.026206385143196727, P69=0.0036901905605843197, P70=0.0005749896532349445, P71=0.00010685571069545976, P72=9.263442751058204e-05, P73=0.00023849245569471184, P74=0.000454500939488235, P75=0.0014903740504041301, P76=0.003385301194676151, P77=0.015303527807987006, P78=0.06617998333899125, P79=0.10736104960237708, P80=0.14600784696260224, P81=0.2598024064366327, P82=0.1839481144831514, P83=0.08829907420020716, P84=0.08863687090653581, P85=0.040698640316063656, P86=0.007284674109404682, P87=0.0016063749136796565, P88=0.001232638400070481, P89=0.004473016650006164, P90=0.033151413099480755, P91=0.08017072908427944, P92=0.08475106797721009, P93=0.039759913582783385, P94=0.09747420715686042, P95=0.0993876574996109, P96=0.041890502697304564, P97=0.03141475428890213, P98=0.007076227448469184, P99=0.011224966682957542, P100=0.03634133752353611, P101=0.016243711172831538, P102=0.0038993780416563674, P103=0.00915905539133925, P104=0.025727948646326392, P105=0.020777436142422043, P106=0.008754734130792748, P107=0.004631987677172266, P108=0.0026598774714917217, P109=0.0024156477568967826, P110=0.018018812264835024, P111=0.030019252217570798, P112=0.02111748255539889, P113=0.01704446498520187, P114=0.002884407750135006, P115=0.003556706390722404, P116=0.004017785461290375, P117=0.008218429684563572, P118=0.0076942106016146695, P119=0.0011863135119330385, P120=0.004291472190878996, P121=0.00586122241344956, P122=0.007212185989498143, P123=0.002075966341636983, P124=0.0025689147952892032, P125=0.0026378452373759854, P126=0.001704300158934456, P127=0.0036799296831165258, P128=0.002770776418759825, P129=0.0003285004634264722, P130=0.0004101512872751323, P131=0.000692568620469518, P132=0.0002438420468473518, P133=0.000467901073527667, P134=0.0008933860062370967, P135=0.00039310154572104965, P136=0.0007329853418415872, P137=0.0008177480234182548, P138=0.002109961965082835, P139=0.0015504930242583672, P140=0.0002918381261374971, P141=0.00028362547661853497, P142=0.00016707373464883328, P143=0.0002857947776192439, P144=0.00019844058563864837, P145=0.0002581940866926357, P146=0.0003596412456109511, P147=0.0004115252874153844, P148=0.00025929149211677185, P149=0.0002347280682891935, P150=0.00014782662092451958, P151=0.0002008973843488797, P152=0.0001271249873289159, P153=0.00029245199442332005, P154=0.0005394264869181306, P155=0.00033693220000869084, P156=0.00033019524637498165, P157=0.000259580144598346, P158=0.00022852817719641, P159=0.000337672705989587, P160=0.0004155801211629827, P161=0.00031288021354085646, P162=0.00043355696844557903, P163=0.0003601581075782403, P164=0.0003411218702460579, P165=0.00028786506738036067, P166=0.0002636601826943157, P167=0.0002850733487807454, P168=0.0002278133160166916, P169=0.0001523302483315731, P170=0.00014888294116911067, P171=0.00015827149067763863, P172=0.00022099376456666636, P173=0.00027673949177858137, P174=0.00017557553065023892, P175=0.00012803707574596525, P176=0.00012898743258405606, P177=0.00018699172839660555, P178=8.40703123647202e-05, P179=6.694158542539388e-06),Row( P0=-642.327882732871, P1=40.80974653458961, P2=-24.825013130613094, P3=-8.443274210923956, P4=-15.100976582194546, P5=-17.007294337230057, P6=-10.330093588338231, P7=-1.805814834349383, P8=-9.086699189158615, P9=-6.857471567674887, P10=-11.477364301961629, P11=-2.727960347833416, P12=-7.242680914793421, P13=-7.09341447068395, P14=1.2571207217372513, P15=-6.391950560783766, P16=2.0205882789758736, P17=-3.8912369365076236, P18=2.8092362358246854, P19=3.0221395446083688, P20=3.3189008268197386, P21=13.499714138143352, P22=10.157643528016362, P23=12.02537300382381, P24=6.701889269019929, P25=3.0284688990495843, P26=-1.6686776886916528, P27=-1.7372145922216469, P28=-1.627658664286989, P29=2.5037222132273014, P30=-1.6844027515529523, P31=0.32477431466215284, P32=-5.2750219598979955, P33=-0.9301047950221932, P34=-2.2981419798992313, P35=-0.5172913931062895, P36=-4.6483623781011305, P37=-3.437309823430857, P38=-2.3777976542202697, P39=0.6314023041274219, P40=0.6658462455560554, P41=0.6539714624358806, P42=0.6559536716071995, P43=0.6462455066361229, P44=0.6188870920212447, P45=0.6653244202031052, P46=0.7012744741351515, P47=0.708105183507296, P48=0.6648171804679779, P49=0.5962653647645182, P50=0.5965059489726401, P51=0.6147704106879911, P52=4.290021703192154e-07, P53=1.5884143103724924e-06, P54=2.8898627898618845e-06, P55=2.674276805990387e-06, P56=1.5148722744780412e-06, P57=1.5582885004899363e-06, P58=3.824175172237177e-06, P59=2.733792305441016e-05, P60=0.00033729261016733907, P61=0.0030597759070137964, P62=0.018955953672488028, P63=0.03149912116228965, P64=0.020306865682720054, P65=0.008926349141613738, P66=0.008367394701386111, P67=0.011652080712717446, P68=0.005198886182300859, P69=0.0009386310323063738, P70=0.000907542141430328, P71=0.0010391184297958566, P72=0.003685076710001438, P73=0.008744432004862562, P74=0.017615358294104697, P75=0.02529026637944868, P76=0.026566655380624495, P77=0.026590787520172358, P78=0.013210899727361147, P79=0.006085337049467686, P80=0.003957560013470371, P81=0.0036873190374750635, P82=0.005143755498158598, P83=0.017139600629214637, P84=0.02319674466361716, P85=0.03603717587076416, P86=0.04555386273779069, P87=0.08474737572354547, P88=0.08977330147902735, P89=0.04263542992241726, P90=0.01611615849964288, P91=0.00853522187458487, P92=0.018938425302506674, P93=0.017851863209894438, P94=0.02928677819095355, P95=0.04383615393633839, P96=0.043601127780090336, P97=0.07687135936924806, P98=0.05282366987461418, P99=0.04297894193155708, P100=0.022998814056892514, P101=0.017232223292987278, P102=0.021187437706443286, P103=0.017959403462860034, P104=0.00867587341897677, P105=0.004382059791255226, P106=0.00305258905788707, P107=0.0036811518501415823, P108=0.00209206931876655, P109=0.0018839655993011897, P110=0.012923712703153371, P111=0.021129622519683516, P112=0.009560701898804213, P113=0.0041935397729285125, P114=0.0036359057867494885, P115=0.0023899729698324693, P116=0.0017032508405385347, P117=0.004450656161502206, P118=0.007729790280637047, P119=0.0043111631158728025, P120=0.0011118664404586285, P121=0.0007489459463926538, P122=0.006306990829379147, P123=0.006606048605636763, P124=0.00044984569431122227, P125=0.0020688961115624275, P126=0.0022710282524531374, P127=0.0003354449312748563, P128=0.00039147594820180523, P129=0.0004149247345658972, P130=0.00025978327125023216, P131=0.0003539098513949821, P132=0.00030573374217209416, P133=0.0007275472328151296, P134=0.0006216334003289029, P135=0.0003972800022397926, P136=0.0010873787660027863, P137=0.0005462242813036388, P138=0.000627376010872388, P139=0.00039861576379429485, P140=0.00017162117967586035, P141=0.00016476337176322864, P142=0.0001576415142616522, P143=0.00024835333140588433, P144=0.0002255954818385839, P145=0.00019448475821775887, P146=0.00010851550193580221, P147=0.00013355732281189524, P148=0.00010508750958288375, P149=8.926829744599457e-05, P150=7.72652725809415e-05, P151=6.0334352435787425e-05, P152=4.88223080408331e-05, P153=4.8340987397266226e-05, P154=3.34575435947625e-05, P155=2.9615370278859732e-05, P156=7.430543507746073e-05, P157=0.00013279504267964167, P158=0.00018021200271509387, P159=0.0002921833287218334, P160=0.00030658726315764453, P161=0.0003816749916970834, P162=0.0005581656589157001, P163=0.0003884620282280661, P164=0.0002062321287980454, P165=0.00018136292953851483, P166=0.00023320583476571863, P167=0.0001696385517379096, P168=0.00013143935560876673, P169=0.00017083616012603163, P170=0.00010260266658711351, P171=6.817998000260841e-05, P172=8.369707898403636e-05, P173=7.677722137096325e-05, P174=7.505524996533774e-05, P175=0.0001028384141821955, P176=7.681740333719166e-05, P177=4.687961662936051e-05, P178=2.050344265165731e-05, P179=1.5779861914446236e-06),Row( P0=-538.0739016737986, P1=26.383353263161784, P2=-30.385297104528764, P3=-13.085898323688363, P4=-18.32139241613277, P5=-23.333547648840913, P6=-13.490964064282025, P7=-7.730800081555617, P8=-11.738531195786086, P9=-9.265614745525967, P10=-9.857844341076534, P11=4.880591707978065, P12=4.426598266164583, P13=2.8580724058510634, P14=8.936490801308093, P15=1.5466522504676556, P16=-0.6142193340689986, P17=-1.5372470298829126, P18=2.9618056028550406, P19=-0.3023037360215347, P20=-0.8084415664932609, P21=3.531595234543362, P22=1.7044431449852773, P23=2.6559334179129483, P24=-0.5973941218472034, P25=-2.18180947690739, P26=-3.6265952400994936, P27=1.5815980240877625, P28=-1.8731255992817986, P29=1.0079318413595426, P30=-3.4392123053375205, P31=0.5802017575775936, P32=-1.340269278342385, P33=-1.3549696149132486, P34=-3.0744203353698736, P35=2.5891366965732145, P36=-0.8753326527686233, P37=-0.008590884327693038, P38=-2.628087532969856, P39=-0.22319727727907498, P40=0.5525283963790574, P41=0.5517022974894441, P42=0.5698974466434636, P43=0.6215539377484187, P44=0.5995067612019568, P45=0.5907809964160786, P46=0.6749738860071695, P47=0.6951392839261908, P48=0.6431615945894975, P49=0.6434569354614976, P50=0.6357000093428676, P51=0.5822639148711, P52=1.069875451070791e-05, P53=1.4767337641440088e-05, P54=3.186414870530627e-05, P55=7.306661676290103e-05, P56=0.00014494635516463576, P57=0.00017765191679531943, P58=8.558499145663563e-05, P59=4.4394951837057085e-05, P60=2.641448912738728e-05, P61=9.638195491754949e-05, P62=0.0032929492155250296, P63=0.010039929651746929, P64=0.015326780753540348, P65=0.05269451404126371, P66=0.08045483066173216, P67=0.08307728614983027, P68=0.12484886619119837, P69=0.2939683366907479, P70=0.47161054673900543, P71=0.29228948538229343, P72=0.1440298049776851, P73=0.2316785867926506, P74=0.3960976701567662, P75=0.4368566179257093, P76=0.38436599023263024, P77=0.34230235102710466, P78=0.07149855461294453, P79=0.017464208025541314, P80=0.01557913202112113, P81=0.008521108029595258, P82=0.005694338977813792, P83=0.009140390639539087, P84=0.014217336928761105, P85=0.017911138661570826, P86=0.04814025907589348, P87=0.11476223427700198, P88=0.2948439522100265, P89=0.4098894375100245, P90=0.27339230131880854, P91=0.14453760817282002, P92=0.12876035914475575, P93=0.10127273984399557, P94=0.3202338719950662, P95=0.22895236364044325, P96=0.1315231208332409, P97=0.15047288898188685, P98=0.1305912262356048, P99=0.18392784746845278, P100=0.15441690690894727, P101=0.07403067074592547, P102=0.01846733035722872, P103=0.011331356930362062, P104=0.018862143090801742, P105=0.05250592599177498, P106=0.28837040381917056, P107=0.36266069814797985, P108=0.062208263980057764, P109=0.03552821218004616, P110=0.031777863882905974, P111=0.025890254168549308, P112=0.013251011804019941, P113=0.009991309453723974, P114=0.016240205176545522, P115=0.07370038520499124, P116=0.14104325413369342, P117=0.058534318503608934, P118=0.028155037515973577, P119=0.009430520365129062, P120=0.007881624479742578, P121=0.01114011865677682, P122=0.05686722876294793, P123=0.041502968852299066, P124=0.007131959308925273, P125=0.00790432329986261, P126=0.0064918868465257, P127=0.017483923719709017, P128=0.01265185015985382, P129=0.00331968657103324, P130=0.0013913809048938168, P131=0.0018559343183082307, P132=0.004510671716831786, P133=0.002722574526509761, P134=0.002942795417139805, P135=0.0022330575328554114, P136=0.003078170365224042, P137=0.0031546595716110837, P138=0.00406174052106571, P139=0.0028379876104312556, P140=0.0030896427551684717, P141=0.001446532725864925, P142=0.000763772532737928, P143=0.0010425355447805855, P144=0.000927404807853545, P145=0.000720662409976579, P146=0.0008081625937235579, P147=0.0007914121742029776, P148=0.0008346561881361816, P149=0.000502228580117177, P150=0.00029050575896480787, P151=0.0003984471108427602, P152=0.00036044290335866634, P153=0.0004754814387659692, P154=0.000727316653734538, P155=0.0007309814456204928, P156=0.0009640347830347893, P157=0.0014791204606361156, P158=0.0008819473803073599, P159=0.00044823427152699977, P160=0.0005965104607986283, P161=0.0006526621163832373, P162=0.0004104637338615776, P163=0.0008015496519585546, P164=0.0006865934728619013, P165=0.0003990354530844216, P166=0.00036710426296863205, P167=0.0002749267920988112, P168=0.00026540401808233067, P169=0.00020019769624332305, P170=0.00016226058594826786, P171=0.00023965771424775661, P172=0.0001910190169291936, P173=0.00019119397815249505, P174=0.00021641727784693244, P175=0.00023957356683331895, P176=0.0003499639380106272, P177=0.00039381940875481505, P178=0.00017129391577700215, P179=2.3847850798885517e-05),Row( P0=-641.7298526708687, P1=45.192198667373404, P2=-32.66855667869746, P3=-7.889920433137172, P4=-15.13218912070581, P5=-17.22046879865569, P6=-9.08025439503736, P7=-5.18256888931629, P8=-9.564133475588692, P9=-5.247617517961307, P10=-12.278223010237216, P11=-5.674280291022036, P12=-2.862527981090555, P13=-8.273138750388451, P14=1.7934397158334359, P15=-4.233491286790052, P16=2.2563775888719446, P17=-1.4142723471443204, P18=1.3536331294599586, P19=3.8991945323302963, P20=4.062137842796612, P21=9.665267733845358, P22=9.2563463040303, P23=9.351044702341333, P24=5.297749814653719, P25=3.0758252523941447, P26=-1.3985255397496792, P27=-1.4808037193295422, P28=-1.2847390212506176, P29=2.4071159031044482, P30=-0.28556175398625083, P31=0.1560032692016227, P32=-4.293904579290474, P33=-1.9185890339553247, P34=-3.9058031903683887, P35=-1.2146373177448868, P36=-1.8353118678251976, P37=-1.770133414052658, P38=-2.7488848168769877, P39=-1.1523779482508771, P40=0.6574693727727172, P41=0.658444607503243, P42=0.6485079052299079, P43=0.6407628334044155, P44=0.6492599512205941, P45=0.7093181337444991, P46=0.7355259073582409, P47=0.6336695073425985, P48=0.6307637552093361, P49=0.6254406639353159, P50=0.6098402443559449, P51=0.6285086948252772, P52=3.045709537977883e-07, P53=1.3518264603486133e-06, P54=2.3523843171222555e-06, P55=1.1821438635370236e-06, P56=6.655962655487735e-07, P57=8.171758193985335e-07, P58=1.8732020756243212e-06, P59=2.1508742166227222e-05, P60=0.00024264915353494108, P61=0.0020248143855951164, P62=0.016703788223575367, P63=0.029699672294630296, P64=0.012536058457176905, P65=0.011271141659642893, P66=0.01504013089894174, P67=0.016780345542558008, P68=0.008347636498126317, P69=0.005316210694769147, P70=0.004173988543876589, P71=0.0017014298780545227, P72=0.0020147089716405554, P73=0.004547007696473192, P74=0.01030449347291162, P75=0.015095546547386209, P76=0.01168999495767034, P77=0.010070019220720221, P78=0.017161265138072385, P79=0.018741301269077762, P80=0.009134692503459762, P81=0.009137520500817245, P82=0.010375529789721763, P83=0.018700903519857745, P84=0.03465585961385663, P85=0.05061602668116766, P86=0.0603923743039823, P87=0.09489333512964218, P88=0.08999518218537629, P89=0.021541475651356033, P90=0.00882294182422396, P91=0.00958837614822215, P92=0.011385278309774845, P93=0.005597431559464847, P94=0.011802527066071298, P95=0.020440386714150285, P96=0.01969226465753081, P97=0.013867774813456295, P98=0.012083033290702918, P99=0.007103991281341427, P100=0.001764227292690814, P101=0.0020235604092703283, P102=0.009003857597372596, P103=0.008611415718776399, P104=0.007596727085473239, P105=0.006484093064262861, P106=0.004810952689441941, P107=0.003140434520192117, P108=0.002021402813268657, P109=0.0012000916274343518, P110=0.007232448006724969, P111=0.014213404450606898, P112=0.006110935268410749, P113=0.001105774269970642, P114=0.0009602230969908, P115=0.0012758129929418016, P116=0.001368244815443627, P117=0.0029263348352034577, P118=0.0009477362230346068, P119=0.0009652732181081407, P120=0.0023663457672488866, P121=0.002714202303609389, P122=0.001994296098861807, P123=0.0013078848159308934, P124=0.0006104723011189109, P125=0.002623025741735809, P126=0.0026695883230459195, P127=0.0016336829235865497, P128=0.00341461875344784, P129=0.0031548197184963545, P130=0.00413913932624022, P131=0.002844875140234788, P132=0.0020643763814413465, P133=0.0012368448905473342, P134=0.0013573686637340801, P135=0.0015769972452716436, P136=0.0028070364867165388, P137=0.002783013076191931, P138=0.001687228559933355, P139=0.0006695908166647735, P140=0.0004682820514540276, P141=0.00038736118377998035, P142=0.0003037442207258311, P143=0.0003399153777845308, P144=0.0003618906363338601, P145=0.000612592039664989, P146=0.0007065294133453802, P147=0.0005701026561988347, P148=0.000310107032456642, P149=0.0001228318479142475, P150=7.81777432398417e-05, P151=4.358899970374745e-05, P152=2.657527436907748e-05, P153=1.7607294138571216e-05, P154=1.696331734668281e-05, P155=1.3808484191098915e-05, P156=1.6195838737881062e-05, P157=2.0666102963733014e-05, P158=3.299416862677292e-05, P159=3.6198686536731824e-05, P160=4.94413010020971e-05, P161=4.242068481719648e-05, P162=4.154777046048517e-05, P163=3.205431758598497e-05, P164=4.6144692955440754e-05, P165=4.539716734966504e-05, P166=2.7848208655226484e-05, P167=2.991769311535926e-05, P168=5.09945869110229e-05, P169=1.937920011202262e-05, P170=1.004078652285988e-05, P171=7.704861975034025e-06, P172=1.1932673327123726e-05, P173=2.4408169127547993e-05, P174=3.8198996148578544e-05, P175=1.3120551453908126e-05, P176=1.865006754750896e-05, P177=1.4987473496061459e-05, P178=5.549704425108938e-06, P179=1.2874437557859682e-06),Row( P0=-526.2153529470015, P1=9.695288670029427, P2=-32.12605615344999, P3=-11.149263418989989, P4=-23.538496598380814, P5=-20.646238978175685, P6=-12.82541269316746, P7=-6.569407211815308, P8=-7.284231687243695, P9=-5.686391004518224, P10=-3.5571763965592464, P11=9.153597800040131, P12=5.4070628768410245, P13=12.037750151701184, P14=11.696721770717145, P15=2.07405397019697, P16=1.5150648572582277, P17=-7.834034435292222, P18=0.8918297154109555, P19=-1.591015658889735, P20=-4.226017639164843, P21=1.7395377389635316, P22=-4.1621566852497764, P23=-1.570489265604627, P24=-0.07919932775895117, P25=1.4769978625534652, P26=-1.5596676399469254, P27=-1.12444318435069, P28=-3.3086970180604207, P29=2.0660448012725685, P30=-2.9812750393590717, P31=1.7807462157294545, P32=-2.345477360731676, P33=2.946990612844435, P34=-0.17852011188711134, P35=2.1902056713766758, P36=-1.6520544483419757, P37=0.3736998999423888, P38=1.2375221941620205, P39=2.2033363048396195, P40=0.6485484785106839, P41=0.6395735937635589, P42=0.6589116881661082, P43=0.655308073832206, P44=0.711276835172451, P45=0.6780111547511121, P46=0.624450065847688, P47=0.5487705005785751, P48=0.5651954763879203, P49=0.618184683185475, P50=0.5961015833266915, P51=0.6332494072829498, P52=1.949857562817136e-06, P53=4.760681025256422e-06, P54=4.625089109685327e-06, P55=8.520699313859179e-06, P56=2.2378726693882503e-05, P57=1.9192810015930878e-05, P58=7.192911482458193e-06, P59=9.604930356691814e-06, P60=8.689094338769423e-06, P61=9.71803463860839e-06, P62=1.2587290142924328e-05, P63=1.738748140130223e-05, P64=3.253032700884687e-05, P65=0.0006081984076626418, P66=0.006850882628927334, P67=0.02170424326355802, P68=0.04356394312534421, P69=0.06804209159019486, P70=0.0978347119365828, P71=0.1962430246565589, P72=0.22665543904789923, P73=0.2816489063202191, P74=0.3029589056357638, P75=0.19417811435653695, P76=0.19768162228601416, P77=0.20623515201741968, P78=0.1000816070512777, P79=0.03174654420814552, P80=0.018011782501903804, P81=0.02291717296715607, P82=0.05799315851586122, P83=0.0711481215908776, P84=0.033298908612355384, P85=0.016591782282745632, P86=0.06170533939536774, P87=0.0783697257571326, P88=0.08197749039721253, P89=0.15267424723496667, P90=0.4614551175531715, P91=0.613075253330642, P92=0.5138457369491671, P93=0.18354448022708167, P94=0.4297291965154124, P95=0.18200727653753718, P96=0.03349506548703479, P97=0.047705032012665155, P98=0.1114484493263736, P99=0.14017873161908628, P100=0.026075569910039155, P101=0.014117968509573586, P102=0.036366429229499925, P103=0.11748639978109024, P104=0.18902515646828746, P105=0.10900099998994216, P106=0.7172863346209724, P107=1.267550462331019, P108=0.10450930629970648, P109=0.020064433626161768, P110=0.06257666982568127, P111=0.17194050588888563, P112=0.09308544789324875, P113=0.009539538417474952, P114=0.01234492062921341, P115=0.02691983560758452, P116=0.02485123061419093, P117=0.012284282763381813, P118=0.018737090792535986, P119=0.025959305795970666, P120=0.025882416496724257, P121=0.03150679941695759, P122=0.030990208544686055, P123=0.010368474840672577, P124=0.05419914478073049, P125=0.026453601601485122, P126=0.010356338219432673, P127=0.006072128202642592, P128=0.0038440264525347397, P129=0.01037853245431111, P130=0.0035180175606576527, P131=0.003859853623925502, P132=0.005418839687915395, P133=0.013153147774050099, P134=0.005411212514097275, P135=0.0055239917733365135, P136=0.009652232734257134, P137=0.008945911997863727, P138=0.003088947162000715, P139=0.0027760655617648966, P140=0.004117919293669449, P141=0.0012512701803923615, P142=0.0005670704120194355, P143=0.0013746342349090742, P144=0.001001691026188102, P145=0.001488152088492354, P146=0.0017436192007108304, P147=0.00140201531327539, P148=0.0023820649866435127, P149=0.005015913146946331, P150=0.0026879245550887375, P151=0.0012215639483294444, P152=0.0006343921070165661, P153=0.0004579564556716874, P154=0.0003309618463244266, P155=0.00033127812932048917, P156=0.0004180189495828923, P157=0.0003893045293191599, P158=0.0006775951677087412, P159=0.0007349642507701777, P160=0.0010181761701007809, P161=0.0008204136290023217, P162=0.0008796947262406836, P163=0.0005223579983360093, P164=0.0005459256917700806, P165=0.0005940879980409221, P166=0.0005344873064724649, P167=0.0007372372049868567, P168=0.000610852734483336, P169=0.00028113979901691054, P170=0.00014170957779871567, P171=0.00012587205175265649, P172=0.00014724118204596665, P173=0.00028559649920252843, P174=0.00024072865121936148, P175=0.00021331470104532656, P176=0.0002560953045340985, P177=0.0002418485159281251, P178=0.00016182953149984058, P179=2.4181321457463537e-05),Row( P0=-712.9241401462057, P1=44.557631115791615, P2=-11.99875677022311, P3=-7.087541199647298, P4=-10.152732582984969, P5=-13.359997580496472, P6=-8.067772932277617, P7=-1.136275956323153, P8=-4.16978882694583, P9=-7.8734513590708115, P10=-8.511199382597756, P11=-3.2675199608211996, P12=-6.314202846950075, P13=-6.499332758002699, P14=3.8070487497021133, P15=2.0056465524938796, P16=2.754813640221547, P17=4.246552062975663, P18=8.199733461627618, P19=1.9297241729951324, P20=4.818283513952598, P21=6.343737276256934, P22=2.5163197998662885, P23=2.2904095756020335, P24=0.39462327583822837, P25=2.1411977008949625, P26=0.9806766417128748, P27=2.7176401197836144, P28=-0.4749661963213436, P29=1.0091718812642156, P30=-3.3099151476828252, P31=-1.468563187287911, P32=-2.51218901731226, P33=0.004154187241772378, P34=-0.8472315913034134, P35=-1.7594147065360737, P36=-3.860124507801289, P37=-0.745900381801781, P38=0.870415375581585, P39=1.558838806338533, P40=0.6722283049752106, P41=0.6151768072132627, P42=0.6072198877883661, P43=0.6234098320962284, P44=0.6668359524505023, P45=0.5970799098217202, P46=0.564560401908201, P47=0.6233235130830516, P48=0.6598669651310046, P49=0.7169285295929556, P50=0.7063705563308131, P51=0.6880552491382934, P52=5.131908747080573e-07, P53=1.7049047012249878e-06, P54=2.864767036191543e-06, P55=3.5408470047103852e-06, P56=3.894791437037627e-06, P57=1.8536625795168991e-06, P58=1.2091686239155308e-06, P59=7.116123714896344e-06, P60=0.0005156247443772735, P61=0.0031204874952141513, P62=0.005010644441791413, P63=0.009400319503136821, P64=0.01366904036733363, P65=0.0282059357982433, P66=0.01669160473503014, P67=0.008933978146450484, P68=0.011756584290423254, P69=0.00672198523120734, P70=0.0009980424902844111, P71=0.0006229398298765569, P72=0.0007452583298632023, P73=0.0009753827050498694, P74=0.0006679505554769738, P75=0.0012249227196981457, P76=0.0022825065661399016, P77=0.0028979713702396823, P78=0.008821512740266439, P79=0.015092921610038565, P80=0.013522669153334511, P81=0.00899181997005198, P82=0.005758833262553235, P83=0.006620387635405677, P84=0.008076185484126596, P85=0.008952559500254165, P86=0.005454112280848003, P87=0.0021569873662822445, P88=0.0019720480934083025, P89=0.0008554527101353788, P90=0.002439604556729544, P91=0.00737731870472123, P92=0.007077794499072364, P93=0.0011814195961434236, P94=0.00039390219488802673, P95=0.00038419616450683324, P96=0.00316795869561537, P97=0.0014101063396122836, P98=0.0019763058088629573, P99=0.012617071938144665, P100=0.005808242594236792, P101=0.0007255946215390775, P102=0.00031761003951038933, P103=0.0016418160171223746, P104=0.0003511162170655866, P105=0.0005639214597583158, P106=0.0035523124617602604, P107=0.0030975034564315667, P108=0.000349677014509265, P109=0.00015785750704950746, P110=9.377914903482604e-05, P111=0.0013380393492195377, P112=0.0029368958679641525, P113=0.002025163130755473, P114=0.0007555845335678468, P115=0.00014951404610747602, P116=0.00033378784434833425, P117=0.0005195244645135142, P118=0.00030620226872124176, P119=6.434091953745384e-05, P120=0.00018432731279219568, P121=0.000500968597683656, P122=0.00018302505780210614, P123=4.7730470917963565e-05, P124=7.448576563790372e-05, P125=0.00010521302391847805, P126=7.37406222001555e-05, P127=0.00012630926943987453, P128=2.7815098087505238e-05, P129=1.200605566562707e-05, P130=2.121508307467843e-05, P131=8.258852534294048e-06, P132=1.0656704196605876e-05, P133=1.880799962096538e-05, P134=1.0847539211727829e-05, P135=2.308965551367851e-05, P136=2.0474449314412273e-05, P137=2.1994204680311745e-05, P138=5.151821008836663e-05, P139=3.386057396189403e-05, P140=3.226152559098764e-05, P141=1.1607729749407696e-05, P142=1.7984324133180446e-05, P143=2.610125919541567e-05, P144=2.8607577673172852e-05, P145=1.792471923156989e-05, P146=1.8108735846366496e-05, P147=1.3560082387262647e-05, P148=1.6084348475376427e-05, P149=1.69722115372463e-05, P150=1.0827116787917063e-05, P151=2.7695820129255088e-05, P152=6.611735834299086e-05, P153=4.2606290895420184e-05, P154=1.700553830219007e-05, P155=2.4319539503615185e-05, P156=4.730207939962103e-05, P157=7.953697471361897e-05, P158=4.5645864291724596e-05, P159=6.23819649528425e-05, P160=0.00011054667767661812, P161=9.517695633675531e-05, P162=0.00010677380146209674, P163=5.600756636035775e-05, P164=5.6781248616235235e-05, P165=4.825714870991414e-05, P166=3.738528882129091e-05, P167=2.1625301979542836e-05, P168=1.0476168401735359e-05, P169=8.07581063089721e-06, P170=2.5323596502488393e-05, P171=4.53371419205269e-05, P172=4.269672430568548e-05, P173=1.3133720917049018e-05, P174=1.1326931546080396e-05, P175=1.3476813179412161e-05, P176=2.489654853852455e-05, P177=3.42940695897622e-05, P178=1.2504660780796907e-05, P179=1.4178033293611553e-06),Row( P0=-625.3401466245876, P1=41.44524558759807, P2=-27.769042694237598, P3=-5.887685012844261, P4=-14.4061004243139, P5=-17.00409976224202, P6=-9.25902963840916, P7=-3.3528792333564192, P8=-5.220124764546469, P9=-8.306562611867513, P10=-12.83855580990608, P11=-0.8399889356086319, P12=-6.217288453694112, P13=-8.154335516928855, P14=1.0410853994080151, P15=-7.197818924026248, P16=-0.15582586948187532, P17=-4.287055680662141, P18=3.8063473136602726, P19=5.099660186178875, P20=4.199617765493773, P21=12.826456043749264, P22=10.447044608410797, P23=10.638149210240718, P24=6.769943362614594, P25=2.389104653835754, P26=-2.256362099365173, P27=-1.429223307195915, P28=-2.7458842076467196, P29=2.5302049554733625, P30=-0.34879147374430697, P31=-0.20294612787017544, P32=-5.779424978134544, P33=-2.1887680907487144, P34=-2.1333712144059187, P35=0.10146954191891892, P36=-1.8970484769441454, P37=-2.874329925999166, P38=-3.6721918708626693, P39=-0.9118462159340105, P40=0.6075321448159919, P41=0.6249014572738666, P42=0.6243666252159212, P43=0.5853808892943804, P44=0.6085770815797932, P45=0.6732795823382937, P46=0.7580594715627024, P47=0.7092029542593556, P48=0.641186586575127, P49=0.6098788965603102, P50=0.6031045129639984, P51=0.5990115845926859, P52=7.056409960367087e-07, P53=3.8219254810534175e-06, P54=5.141618166002208e-06, P55=1.7168893442892212e-06, P56=2.840961163802742e-06, P57=4.99417296846738e-06, P58=3.9728016989642355e-06, P59=1.8225046353175454e-05, P60=0.0003033081782447395, P61=0.0024628362950980627, P62=0.017802657842540617, P63=0.047138220640245566, P64=0.034814147651765145, P65=0.018983443092803143, P66=0.020035161281230146, P67=0.021033250846660894, P68=0.005335615887791336, P69=0.0003265478728634393, P70=0.00039151725799979235, P71=0.0005561537888762401, P72=0.001979025406595204, P73=0.006568816399768916, P74=0.015097171633555853, P75=0.032825586736979086, P76=0.03558330559071889, P77=0.02722699129908565, P78=0.033938655479559356, P79=0.022398328267993033, P80=0.009635821590488548, P81=0.011970287423293272, P82=0.014298966270275795, P83=0.017615516684152947, P84=0.01284487480129332, P85=0.01530568579853503, P86=0.04448811431093785, P87=0.08957894761609334, P88=0.13554777469883508, P89=0.028658943155735153, P90=0.02105872802181571, P91=0.03941285719637967, P92=0.040139189968923565, P93=0.050556800754408354, P94=0.1162206844433977, P95=0.1136803443866566, P96=0.05876420944468218, P97=0.012551933327840192, P98=0.0014088320904525452, P99=0.02013956884192388, P100=0.026551776564143304, P101=0.020575998041901987, P102=0.03343045239456964, P103=0.04753737793671514, P104=0.017605116498942378, P105=0.003099182093951783, P106=0.005326678081969625, P107=0.010279721052199967, P108=0.005855043852904781, P109=0.006819661503904507, P110=0.023605983634366737, P111=0.030070373336549966, P112=0.016583893869464152, P113=0.0036389626812646375, P114=0.0019044772643805991, P115=0.0039344949073726486, P116=0.004843204411224832, P117=0.009088896324484598, P118=0.006262238780235685, P119=0.0008362953876096076, P120=0.002292240322181323, P121=0.004050358874373014, P122=0.0023524451696617663, P123=0.0018056379423011155, P124=0.002830261602313174, P125=0.0007510418475102806, P126=0.000997674392077132, P127=0.001204681467043093, P128=0.00031197515123206353, P129=0.0004418022212302597, P130=0.0003229948399557696, P131=0.00026971314648595063, P132=0.0002823423953709351, P133=0.0002903257620484221, P134=0.00017819029681884565, P135=0.0006906065188552551, P136=0.0008589450416481283, P137=0.0008239592115140191, P138=0.0017835447873115558, P139=0.0025390166063908393, P140=0.0008640115811683262, P141=0.0008466853613402852, P142=0.0005322149175018047, P143=0.000971578119542368, P144=0.0005735489044648908, P145=0.0006674323113714265, P146=0.00029014694601111994, P147=0.00017250529666303587, P148=0.00025827822030212395, P149=0.00030568674883343835, P150=0.00014153038707965168, P151=9.849899537091014e-05, P152=7.583754150784082e-05, P153=6.571777209486544e-05, P154=8.125705804702274e-05, P155=6.963158066375216e-05, P156=5.761675110933253e-05, P157=4.293173440704822e-05, P158=0.00010717432359126918, P159=0.00018148963028463734, P160=0.00014567036540979623, P161=0.00015804915524532533, P162=0.00014343771853308668, P163=0.00011927122020936489, P164=0.00010165600769747183, P165=9.273946917198957e-05, P166=7.026905365251467e-05, P167=5.653653281073619e-05, P168=5.216200612510309e-05, P169=5.4463327276463075e-05, P170=4.544443296092725e-05, P171=5.567639417093013e-05, P172=8.308933664803119e-05, P173=7.052589490029526e-05, P174=5.1884286977891876e-05, P175=5.602590950123382e-05, P176=4.4345826397633975e-05, P177=4.5549005505625057e-05, P178=1.5824724492819002e-05, P179=1.5385556792099484e-06),Row( P0=-664.2120300192482, P1=38.18440232627722, P2=-19.560453999248516, P3=-10.854839344247939, P4=-15.032393504302464, P5=-17.5926718372144, P6=-10.12494006271797, P7=-3.940303846286247, P8=-5.481136736274537, P9=-8.362731180817464, P10=-11.849342595546485, P11=-2.5167876101031372, P12=-7.5234697999125215, P13=-5.497907391152285, P14=2.8512898035574588, P15=3.4193961067862695, P16=8.701434098123997, P17=6.9315770338985265, P18=12.24230806262442, P19=4.355453292137123, P20=1.8970795464816088, P21=3.7855996499875717, P22=1.0836818350480675, P23=4.2179559211183815, P24=1.7540243670805375, P25=-0.16517951705584713, P26=-5.3627901087641625, P27=-3.377387264268411, P28=-4.4986629179482485, P29=1.7453291813477692, P30=0.9749808246324863, P31=0.41160450013057986, P32=-3.21918653593697, P33=1.1992815335246465, P34=-0.5844202776960745, P35=3.3756476572957266, P36=-1.6360298105379925, P37=-1.9349261734259722, P38=-1.4066989373697456, P39=0.5617841753989385, P40=0.7115631224807026, P41=0.6108835716866168, P42=0.5963462273029734, P43=0.5979058547867593, P44=0.6132250100412534, P45=0.6724865197019687, P46=0.7086189601446807, P47=0.6553551228345761, P48=0.6608765385565507, P49=0.6927278788981733, P50=0.7293714431678869, P51=0.722577726903091, P52=5.405988455733813e-07, P53=2.1087982261144428e-06, P54=2.8864883475195243e-06, P55=2.6589277510191002e-06, P56=4.1332119581746735e-06, P57=7.65458215359735e-06, P58=5.310514714672205e-06, P59=3.348477787657009e-06, P60=1.537153863557078e-05, P61=0.00031381687222677254, P62=0.006501953078204055, P63=0.014733629448918982, P64=0.01166692337627625, P65=0.02527778914258248, P66=0.04084459766137629, P67=0.05074671759859046, P68=0.03516017945797456, P69=0.008307331860601123, P70=0.0012615079398576308, P71=0.0002707401824327795, P72=0.0002594631682598194, P73=0.0013033780454451227, P74=0.002671009706009014, P75=0.004993827613916974, P76=0.005106317287411168, P77=0.011214144987669272, P78=0.016081992310069123, P79=0.01924544262954754, P80=0.016797173591492458, P81=0.0148886493981005, P82=0.017366770618430612, P83=0.04692712975043863, P84=0.09050283696412574, P85=0.07124551253540304, P86=0.02768994480349176, P87=0.011958560711735169, P88=0.005133213948684986, P89=0.003987961144173714, P90=0.003959284991977256, P91=0.004688628768488751, P92=0.007072769982477683, P93=0.01754878061112078, P94=0.05024098172529065, P95=0.038104207411831084, P96=0.01749569054166951, P97=0.00664188273672383, P98=0.0019194895123286141, P99=0.0007205135026565687, P100=0.0006294607372323421, P101=0.0022574529403556185, P102=0.02211110266050393, P103=0.01156453922171014, P104=0.0038146541126057466, P105=0.0050994293577417285, P106=0.001613972793995847, P107=0.0003431026111533659, P108=0.0007548934963721708, P109=0.0035759110484609, P110=0.0046897523931352025, P111=0.0038852280762689696, P112=0.0011695281807677063, P113=0.0018137700932531091, P114=0.002769390467255047, P115=0.0056888591590339285, P116=0.002877883236144209, P117=0.003911438544189399, P118=0.000839066885229791, P119=0.0006766608322304586, P120=0.0025756415427907148, P121=0.0020088165088476277, P122=0.0006451488751074932, P123=0.0010974184554707125, P124=0.0008411040302563, P125=0.00023949179602086518, P126=0.00023412064895826388, P127=0.0010124994580677498, P128=0.00031917186414446186, P129=0.00021360277776230255, P130=0.00010150299166220693, P131=0.0001304388149044424, P132=0.00019245091766119238, P133=0.00011609999323973727, P134=0.00014106954500851015, P135=0.00016378807922520406, P136=0.0004522640189487931, P137=0.0002820839676644134, P138=0.0003045079042574911, P139=0.0003276036166483149, P140=0.0005257011216421216, P141=0.00014029309948196742, P142=5.04064426502516e-05, P143=6.547112605030355e-05, P144=6.926984634951386e-05, P145=0.0001445741614143525, P146=8.95971111476729e-05, P147=6.358869607418609e-05, P148=7.716385665143781e-05, P149=9.916005727758276e-05, P150=0.00011385557269369241, P151=7.707197331513129e-05, P152=8.802673309370437e-05, P153=0.0001861902477186377, P154=0.0001591170744456115, P155=0.0001231510612470315, P156=8.674753807586087e-05, P157=0.00011796023522692522, P158=0.0002620677094479961, P159=0.00038903750629760205, P160=0.0003421900735387726, P161=0.00019972605061682768, P162=0.00017294642780158698, P163=0.00019602960563879025, P164=0.00015936726266112887, P165=0.00010988724261677055, P166=5.5469564398849766e-05, P167=3.541848341828204e-05, P168=2.038977561799368e-05, P169=1.9482489284095195e-05, P170=1.672413465259618e-05, P171=2.008635020725646e-05, P172=1.8630033640699966e-05, P173=2.447408097441725e-05, P174=6.053022503290066e-05, P175=9.476196843461489e-05, P176=8.394907741441972e-05, P177=8.182295717824491e-05, P178=3.370193504037959e-05, P179=3.3511688695757552e-06),Row( P0=-710.5231047962529, P1=44.13974480162258, P2=-10.519245575695823, P3=0.24592615947049928, P4=-8.066585372697558, P5=-10.08007366901587, P6=-5.384008863963829, P7=-2.7968690895980637, P8=-4.4275447705688045, P9=-5.068841494493887, P10=-8.24395173277297, P11=-3.831346321306532, P12=-6.2758789084343025, P13=-7.692894739711545, P14=0.6538756045642812, P15=0.04150794512589826, P16=0.264023451932858, P17=0.06656532447583459, P18=8.02691894758579, P19=1.1855025859012163, P20=3.4472055351684205, P21=5.352496191253882, P22=4.595002898452283, P23=6.310082904404733, P24=4.9371811263204, P25=5.187067324850539, P26=-1.6927336850349115, P27=0.5290286264600155, P28=-3.0114094935437805, P29=2.9445959262463157, P30=0.01919739090037371, P31=1.754885668676101, P32=-2.261893352567901, P33=-0.174568594810466, P34=-1.3856998803119402, P35=0.6278447018374629, P36=-4.543768205898391, P37=-1.515201288183091, P38=-1.8824497842986243, P39=1.8646807150299296, P40=0.7029981849163047, P41=0.6134471545778952, P42=0.5405130290022025, P43=0.5992353920450975, P44=0.6250896941095787, P45=0.6630169821992681, P46=0.6822128576782591, P47=0.629900678469179, P48=0.6334837797857465, P49=0.6434934371451979, P50=0.6498631596569153, P51=0.6641838922345203, P52=6.309645194741751e-07, P53=2.2602571647369282e-06, P54=2.6493563299478518e-06, P55=2.2912810691587944e-06, P56=2.5566310080668555e-06, P57=2.6185213228939375e-06, P58=2.8652440994401116e-06, P59=8.858236470707042e-06, P60=0.00034997850596484546, P61=0.002846525031942978, P62=0.012235881487906226, P63=0.014029124169913027, P64=0.005391300704998244, P65=0.003597818687532033, P66=0.008416665891704197, P67=0.010279292224620467, P68=0.00583346936855453, P69=0.0005483505094079722, P70=8.290544169175744e-05, P71=0.00017697121255262144, P72=0.0004796929291453659, P73=0.0009228433821660715, P74=0.0016967004057402342, P75=0.000831620596346953, P76=0.0005793555374016209, P77=0.0008435819745974939, P78=0.0003464035770701569, P79=0.0005823459239712163, P80=0.0016304392751917487, P81=0.002874492787010554, P82=0.0031569305571802395, P83=0.006580375054578739, P84=0.018469263462604014, P85=0.010294468217793541, P86=0.0006684003995180481, P87=0.00010687261220950336, P88=0.0004167696710559508, P89=0.00047560322797236875, P90=5.7900990182331546e-05, P91=3.226229176702705e-05, P92=8.202310032099894e-05, P93=0.0002486169910361854, P94=0.0015795654098541942, P95=0.0006140433931866962, P96=0.0001835460832447715, P97=0.001719546967762386, P98=0.001199752203381591, P99=3.4122233293492214e-05, P100=2.5126819463733442e-05, P101=0.00010715090929552047, P102=8.248605177185995e-05, P103=0.0001957068793253291, P104=0.003926931997924974, P105=0.0016507686413123816, P106=7.591267846857405e-05, P107=5.540523494176273e-05, P108=1.2187521035843216e-05, P109=0.00024133149869114893, P110=0.000953393102416401, P111=7.165908862275377e-05, P112=2.4732970226309558e-05, P113=1.2446355929285996e-05, P114=5.242046030910048e-05, P115=9.575258954550466e-05, P116=5.958549138872359e-05, P117=3.963830179647195e-05, P118=5.372106828924805e-05, P119=5.3318323874733314e-05, P120=9.442032931864478e-05, P121=7.359445487367466e-05, P122=9.180606798859878e-05, P123=0.00014097903708259584, P124=0.00013081227928759376, P125=9.18875764555446e-05, P126=0.00012361529576637693, P127=7.3969689795894e-05, P128=6.617461207281426e-06, P129=9.032428911815862e-06, P130=0.00010751063926166885, P131=3.35080949984359e-05, P132=1.0657024530890968e-05, P133=2.7064459759672736e-05, P134=2.82311391193116e-05, P135=1.5892867031436552e-05, P136=4.062028802789164e-05, P137=4.2992328537162005e-05, P138=3.245164094083597e-05, P139=5.816930744021256e-05, P140=2.02644425033493e-05, P141=2.0693983272637902e-05, P142=2.0770001474299794e-05, P143=2.0654653990090075e-05, P144=2.1262452223183848e-05, P145=1.9080468801328663e-05, P146=1.5321920993053487e-05, P147=1.5144537103210973e-05, P148=2.6904667248381e-05, P149=3.5843809560556865e-05, P150=3.700630918679621e-05, P151=4.4957090962004276e-05, P152=0.00011468819994338738, P153=4.4773534807083344e-05, P154=1.579214333567637e-05, P155=1.7458577950441277e-05, P156=1.9190288542127506e-05, P157=3.035899380765276e-05, P158=2.4810438386762595e-05, P159=2.8335842313278428e-05, P160=3.244370608422616e-05, P161=5.471162835495802e-05, P162=5.4439381451202166e-05, P163=1.6614385617431467e-05, P164=1.4632037857542106e-05, P165=2.0840574898726918e-05, P166=1.4047610297863772e-05, P167=1.0447608708264696e-05, P168=6.9536899485097575e-06, P169=7.748500927734166e-06, P170=9.626089639846445e-06, P171=8.852802336188806e-06, P172=5.683126541675541e-06, P173=7.152159565136407e-06, P174=9.84423418453026e-06, P175=2.1354940066574256e-05, P176=1.6659760429030428e-05, P177=1.506310337476877e-05, P178=4.2526884866069405e-06, P179=5.319647405268951e-07),Row( P0=-709.099298727311, P1=43.62039289035349, P2=-9.517758489802313, P3=-3.767345650193345, P4=-10.755709690016316, P5=-9.410125452316358, P6=-5.056121766286451, P7=-1.8898225272974867, P8=-5.650363510818943, P9=-5.241928562106911, P10=-7.634442482329091, P11=-3.130316809247467, P12=-7.493492604218808, P13=-5.417976080927184, P14=1.8331980269488062, P15=-1.9528539628340063, P16=2.4023784290728227, P17=-1.0997052355065495, P18=5.501155950396767, P19=1.7615641431280924, P20=2.5469013414626964, P21=6.21111576011836, P22=4.816967857875393, P23=8.178275320376363, P24=6.117121010984849, P25=3.1497728850572164, P26=-1.183847332596929, P27=0.30765316621949285, P28=-1.3789100680975706, P29=3.1576390860335435, P30=-2.2433122548245046, P31=0.5878782170118, P32=-3.5485793821600637, P33=0.7770960602301181, P34=-1.0381399014993777, P35=-0.7239339174608013, P36=-4.735849373881851, P37=-2.4839478206825443, P38=-1.2403048695249066, P39=-0.5828978730357004, P40=0.7397955349854338, P41=0.6543470466764189, P42=0.6102254692323995, P43=0.647474303366247, P44=0.6654057309730578, P45=0.6637659082058782, P46=0.6604562759927323, P47=0.6350110198297807, P48=0.6499832504274865, P49=0.6147883756477904, P50=0.6211846151481496, P51=0.6783069388747754, P52=6.912144691498225e-07, P53=3.177870766786869e-06, P54=5.052070652019609e-06, P55=2.6149640119179486e-06, P56=3.2590090394476056e-06, P57=2.7417471665429864e-06, P58=1.4565187627770547e-06, P59=4.57811852334328e-06, P60=0.0005005301500763095, P61=0.0043924478896836, P62=0.012661446014559442, P63=0.016642382557525374, P64=0.007612576363860516, P65=0.00271063811183481, P66=0.003798922478520256, P67=0.010698928087091941, P68=0.0076388832586327315, P69=0.0009993511632115618, P70=0.00020361216732651616, P71=0.00044930544216056125, P72=0.001277063522933596, P73=0.0013813066593233374, P74=0.0029147557198270354, P75=0.0030240740827627006, P76=0.002520387595565378, P77=0.002815476674046699, P78=0.0010645504158416936, P79=0.00038952562251209113, P80=0.0002828476418287228, P81=0.00042876399383548816, P82=0.0017392657322234525, P83=0.012060893514436144, P84=0.02296319882034008, P85=0.009750755916628844, P86=0.002909506484667965, P87=0.0020263958368011677, P88=0.0016151254906761347, P89=0.0012540364288400041, P90=0.0006056108138771828, P91=0.00030054817771737444, P92=0.000264168515067743, P93=0.00024080036946220402, P94=0.004537500425303664, P95=0.003684939464584782, P96=0.0007880474400347741, P97=0.004034187045850438, P98=0.004440996609613526, P99=0.0013659702916610577, P100=0.00022071437699456897, P101=0.0009864946556415284, P102=0.0003962111813910923, P103=0.00026673165450423106, P104=0.004646062492245618, P105=0.0054813140078127055, P106=0.0021611488235357143, P107=0.001188499990809543, P108=3.8924359026640925e-05, P109=0.0005819374929863061, P110=0.003931940980301469, P111=0.002664444842064721, P112=0.0003578247820938208, P113=2.5091832252174238e-05, P114=0.00033538668693431305, P115=0.0007533035918907486, P116=0.0004570834078094573, P117=7.82318661207968e-05, P118=0.00010766952674796713, P119=0.0005812091801912012, P120=0.00029543359800938686, P121=2.9400501776339354e-05, P122=0.0002543746597541071, P123=0.0003451721249963773, P124=4.888697996727846e-05, P125=6.565914023166183e-05, P126=0.00012951537039228333, P127=6.847283615570805e-05, P128=1.3797834346924766e-05, P129=1.8330678811778206e-05, P130=4.806660247539282e-05, P131=1.558928386199786e-05, P132=1.3529637089802843e-05, P133=2.6097240629658083e-05, P134=1.609548299402243e-05, P135=3.0143570079602268e-05, P136=2.8174850948673662e-05, P137=2.284648279817231e-05, P138=6.800812832189447e-05, P139=2.4771964122265822e-05, P140=2.1187438273440213e-05, P141=1.6494660296602136e-05, P142=1.3397113386450237e-05, P143=1.8094660294871632e-05, P144=1.881484188956158e-05, P145=2.6830561432176545e-05, P146=3.4505178370805176e-05, P147=4.2792576193437675e-05, P148=3.69919025088458e-05, P149=4.4643781070970094e-05, P150=4.2945963817062656e-05, P151=1.558138508785488e-05, P152=1.627725022733221e-05, P153=2.145057581578977e-05, P154=3.3957874422019724e-05, P155=2.632505348741485e-05, P156=2.3543955512341963e-05, P157=2.779523205378262e-05, P158=2.036044592866498e-05, P159=2.2562759710339087e-05, P160=2.2704589352033394e-05, P161=2.7026315231724924e-05, P162=3.160866769917703e-05, P163=2.3511186051865496e-05, P164=2.5874250022321565e-05, P165=2.646060142034781e-05, P166=2.7554105806039756e-05, P167=2.5130240169582354e-05, P168=1.3176453420081985e-05, P169=8.166660164143789e-06, P170=1.4097903153492237e-05, P171=1.4887763341413442e-05, P172=1.2368595505908297e-05, P173=1.1903221803696567e-05, P174=1.0531278397760933e-05, P175=8.874533384227457e-06, P176=9.87295435607511e-06, P177=1.5213596356839795e-05, P178=7.991392702283647e-06, P179=7.981247505161787e-07),Row( P0=-536.946851178033, P1=19.908375252891645, P2=-38.314656912523574, P3=-14.680291484435948, P4=-20.137759132216285, P5=-21.819330384901143, P6=-12.988891670643257, P7=-8.453704428403881, P8=-9.378257966415191, P9=-4.310951848856766, P10=-6.991447938983877, P11=8.16037706457219, P12=3.8641246129557643, P13=0.4948942506503109, P14=2.24837840874646, P15=0.33154152617895655, P16=2.6265605211790213, P17=0.07717172583705624, P18=5.722206821001386, P19=-0.5905075356274333, P20=-0.502892875502025, P21=4.611814764945853, P22=0.5551638179048289, P23=0.3183916407244051, P24=-3.570242229796609, P25=-4.035110539509918, P26=-3.2141950305796856, P27=1.3833765073721127, P28=-3.976144299397175, P29=-0.6449462707426181, P30=-3.2120788451404594, P31=-0.11716950250157426, P32=-1.9779362463142733, P33=-1.113190382372645, P34=-2.013364296694062, P35=2.70473796229442, P36=-1.5748109220169255, P37=-0.16735119960199646, P38=-0.9732885905281954, P39=1.6171312142191565, P40=0.5944437265955541, P41=0.5619957264608112, P42=0.5831138877477537, P43=0.6079105746018453, P44=0.5842882093266218, P45=0.5982650780516646, P46=0.6579622559118942, P47=0.6966689170902479, P48=0.7071041791785229, P49=0.6935640207262854, P50=0.6500331170960721, P51=0.6023058314436198, P52=7.56585842413874e-07, P53=1.3632548144424383e-06, P54=3.3489944318651496e-06, P55=1.0199218018107841e-05, P56=2.3502479933173772e-05, P57=4.998210290415001e-05, P58=2.3201458444388405e-05, P59=1.4008342709632244e-05, P60=2.1333975914586167e-05, P61=2.2337360641377313e-05, P62=0.0010013094397841463, P63=0.007125725220640847, P64=0.011381664586987063, P65=0.028333688047595278, P66=0.07843038466518208, P67=0.11701465099285017, P68=0.12781733017027183, P69=0.19903386740767184, P70=0.2513784490820828, P71=0.16761301931573167, P72=0.0974329816343049, P73=0.1237522767539654, P74=0.16673839293158227, P75=0.16558455016182025, P76=0.14419037584699257, P77=0.2629898876306735, P78=0.4525750298514796, P79=0.4486599526279859, P80=0.3494106292538715, P81=0.09537877790631466, P82=0.016056125211045012, P83=0.03653277254928959, P84=0.05613298439404348, P85=0.040324759006183326, P86=0.0444048377882898, P87=0.13418848403626918, P88=0.23717545170851814, P89=0.1548897354900423, P90=0.15686948582169286, P91=0.07492369093353624, P92=0.07168494851757382, P93=0.06696469657186724, P94=0.18134312545671066, P95=0.22351884055170376, P96=0.2587897079619896, P97=0.26539905077571035, P98=0.2084878788225831, P99=0.2505197042013461, P100=0.3146407536659747, P101=0.27532995600320054, P102=0.19708593060669782, P103=0.020540691053018898, P104=0.007391082468442365, P105=0.018370246608433543, P106=0.031013029746635477, P107=0.04463542926245759, P108=0.14704188449173733, P109=0.2892369320756629, P110=0.5649398392948896, P111=0.5209309080422846, P112=0.08967410540881952, P113=0.031487390425309285, P114=0.04870264793751132, P115=0.06430928395372904, P116=0.20179988009639627, P117=0.20139717538370908, P118=0.07931991797041058, P119=0.0265121226317698, P120=0.024802926385064997, P121=0.016500395336390347, P122=0.027731050142908983, P123=0.06374354404113179, P124=0.03252059475507824, P125=0.011018745712579625, P126=0.008134098060234405, P127=0.017970230032892616, P128=0.014813917855633046, P129=0.007775367059062598, P130=0.0023729594279230807, P131=0.0027042576597539575, P132=0.006971995598467431, P133=0.0038384977863503878, P134=0.004572174472038994, P135=0.00888258567140346, P136=0.010233636179628795, P137=0.005840948235135519, P138=0.008635494005441054, P139=0.013791020657378188, P140=0.006707189236870195, P141=0.002439618528837182, P142=0.0017833106694066143, P143=0.0011076762912427278, P144=0.001009755230723509, P145=0.0012647011928798029, P146=0.0014123479251290385, P147=0.001984528070577106, P148=0.0011412896567317202, P149=0.001647556582109434, P150=0.0010634820996527974, P151=0.000439479822161115, P152=0.00040060844775263537, P153=0.00045215886727253154, P154=0.0003348504643261087, P155=0.0004719068930124035, P156=0.0004957411362248503, P157=0.00038575147193631364, P158=0.0009468643045478974, P159=0.0003472840972696601, P160=0.00034232576527073576, P161=0.0003977194524102163, P162=0.00029965866045636246, P163=0.0004074906853845592, P164=0.00038665504579982803, P165=0.0003381900990566979, P166=0.0002989635359215144, P167=0.0002228494525416203, P168=0.0001711704537897286, P169=0.00019010009373745396, P170=0.0001363759868683935, P171=0.00011572743946402482, P172=8.729179567309462e-05, P173=0.00011036310853487653, P174=0.00016878072441804003, P175=0.0001955783161002645, P176=0.00026000585689706746, P177=0.0002409366285318731, P178=0.00010148030061832946, P179=8.822925149850411e-06),Row( P0=-699.4152302377298, P1=50.0465370704302, P2=-15.928726721858883, P3=-4.2686767302000055, P4=-15.31659137189092, P5=-12.800449607385866, P6=-10.822026798077713, P7=-2.874758753407348, P8=-1.009507528361849, P9=-7.316426895858694, P10=-10.2137523099387, P11=-2.156809525596715, P12=-6.275059098132625, P13=-8.082314855834847, P14=0.6149244408520106, P15=-1.8308151832483601, P16=1.515989143275458, P17=3.2376511921548343, P18=9.851536860057841, P19=4.065740700092195, P20=6.7052065529798215, P21=5.453286150104725, P22=0.5815773190876095, P23=2.5272004027818973, P24=-0.5376628639524361, P25=2.870949765775602, P26=1.8333850873295043, P27=3.668042726191107, P28=0.5638352659971534, P29=1.8096047807724351, P30=-1.609823978988737, P31=-0.16347996911150858, P32=-2.718691960150798, P33=-0.056898168578326136, P34=-2.7259174296419846, P35=-0.8168469737963203, P36=-3.4980024667601555, P37=-0.6028693279648882, P38=-0.5188806787870628, P39=0.9306099900501908, P40=0.6259208938393328, P41=0.5935025474255501, P42=0.5895526371692672, P43=0.6298164920237049, P44=0.5991331564170506, P45=0.5861930016670313, P46=0.600709228906882, P47=0.6233204204918481, P48=0.7047048185346899, P49=0.73567630338008, P50=0.6879340525619331, P51=0.6409727137803187, P52=4.2425905566752975e-07, P53=1.7619912746072836e-06, P54=2.7782357004991237e-06, P55=1.156437723829941e-06, P56=7.924202646118271e-07, P57=1.074456956011794e-06, P58=7.833071656237664e-07, P59=6.2115226471475636e-06, P60=0.0003462001300251747, P61=0.0017276073674039134, P62=0.005478525778054921, P63=0.009508186538829063, P64=0.012561042748092333, P65=0.01148463896194838, P66=0.006226748584253051, P67=0.004436134185231188, P68=0.0011743767368567143, P69=0.0001687448707157293, P70=0.0001764555389660038, P71=0.0004346248797901148, P72=0.0005315976354317157, P73=0.0007939529660147964, P74=0.0017955762280877953, P75=0.0017702410416718744, P76=0.0014543931779801194, P77=0.004983605344239623, P78=0.012594389242760785, P79=0.010789801445964411, P80=0.0036418916965611607, P81=0.0016006848364819087, P82=0.0027381840078383305, P83=0.00292116146202469, P84=0.0015785518879627282, P85=0.0014345697273900275, P86=0.001435360385039533, P87=0.000974443778054942, P88=0.0005209642039084181, P89=0.0004679979755942679, P90=0.001616174135002208, P91=0.0023812544441224564, P92=0.001071008414018497, P93=0.0005617776243458993, P94=0.002629813203544331, P95=0.0014920408475293764, P96=0.00028912199717555126, P97=0.00010048503746814847, P98=0.0010949676763505607, P99=0.006379104785143812, P100=0.0015167405552995534, P101=0.0007804440567356647, P102=0.0005846130676637689, P103=8.984735151917871e-05, P104=3.6748337419734414e-05, P105=0.0005008397660503655, P106=0.0018780823899052046, P107=0.0011964161639329042, P108=9.946097162072862e-05, P109=1.647351905984722e-05, P110=6.318045583284983e-05, P111=0.0008876294902528336, P112=0.0010537864210620593, P113=0.0006718064140451953, P114=0.0001495703693703199, P115=0.00011935877947958945, P116=0.00037844143136425256, P117=0.00014043674340795167, P118=9.843557098359424e-05, P119=0.00016109789696589925, P120=0.00025098247224634285, P121=0.00022035951355346838, P122=0.00016189614554725117, P123=9.085995347793188e-05, P124=3.479956971092219e-05, P125=0.000118648790776273, P126=5.026335276089551e-05, P127=1.947507288940391e-05, P128=1.7130409038136604e-05, P129=8.819007481714809e-06, P130=7.84809924119e-06, P131=1.0618190521652075e-05, P132=1.7052406420926754e-05, P133=1.8458561863203887e-05, P134=1.2499404217146173e-05, P135=3.2003632892676335e-05, P136=1.7845087744290494e-05, P137=2.1844167246143187e-05, P138=4.9555469533239616e-05, P139=4.815043267564768e-05, P140=7.889431803148175e-05, P141=3.560409975007565e-05, P142=1.694072461408063e-05, P143=2.2467910659188774e-05, P144=2.0583324710900335e-05, P145=2.9792448170034767e-05, P146=2.7232103185166486e-05, P147=1.9321469843551266e-05, P148=2.2528225013025792e-05, P149=3.54924353881022e-05, P150=3.923801350746517e-05, P151=1.8060001174161866e-05, P152=4.153709223545074e-05, P153=4.3273940611336694e-05, P154=2.575040479519006e-05, P155=2.434102692245989e-05, P156=2.2699526082408726e-05, P157=3.595563173560632e-05, P158=4.529809338046259e-05, P159=4.463929851476612e-05, P160=4.848327131516242e-05, P161=0.00011593034109049965, P162=0.00011603986944700572, P163=0.00018195526989663716, P164=0.0001564500827242037, P165=0.00010706957181681279, P166=8.613557156907775e-05, P167=4.7774920582742944e-05, P168=3.0173767508577635e-05, P169=3.573771583403058e-05, P170=4.645478744652607e-05, P171=3.533546790250302e-05, P172=4.199284631768102e-05, P173=2.4730527246976556e-05, P174=3.236850548125289e-05, P175=3.2293118054527516e-05, P176=3.445796848341046e-05, P177=1.9278190323620698e-05, P178=8.202238507921084e-06, P179=7.520505413899525e-07),Row( P0=-669.0026068080106, P1=40.30324339223932, P2=-18.44845805695029, P3=-9.998952523916874, P4=-9.657827888054726, P5=-16.518971349973178, P6=-10.801937196424317, P7=-2.0488388017263253, P8=-4.250967217175414, P9=-6.9505501827483815, P10=-9.874975109270466, P11=-1.3848639671811829, P12=-7.440371716774115, P13=-5.823811337306396, P14=1.0318283575458544, P15=-2.4531650131212674, P16=1.4030022565623514, P17=-0.3534004785110867, P18=3.7896254202568365, P19=-0.34091770413639266, P20=2.0322411503857767, P21=7.395103383818688, P22=7.0285015861056825, P23=9.274238809468823, P24=6.629717574571102, P25=4.394803656858461, P26=-0.1669361616847268, P27=0.7917283966991235, P28=-2.3875380860305206, P29=2.830862950553007, P30=-0.17463399038016325, P31=-0.15639856311746042, P32=-2.7502648411426325, P33=-1.0735608763509283, P34=-1.1679663829863889, P35=-0.33298072164591136, P36=-3.922058529504794, P37=-1.3017355029369193, P38=-2.4611454507943, P39=-0.7651202991474642, P40=0.660909251736489, P41=0.6515319367078014, P42=0.616190126484991, P43=0.6020384595616346, P44=0.6579765065985524, P45=0.6528271600139184, P46=0.6599206137372681, P47=0.6686341574851924, P48=0.6522704967437595, P49=0.6489645026197249, P50=0.6307084245459637, P51=0.636533854168866, P52=7.042800040441182e-07, P53=2.1653964021190936e-06, P54=2.386010219332867e-06, P55=2.4690996319567307e-06, P56=2.6577295181215117e-06, P57=3.3154631855082804e-06, P58=1.6053361955558537e-06, P59=1.247175952708244e-05, P60=0.0005184567971012693, P61=0.0055833808853513614, P62=0.026476313832094443, P63=0.05294072598037233, P64=0.024959901381478244, P65=0.013891309281925984, P66=0.01211802225313752, P67=0.014375540545318262, P68=0.012920989852629003, P69=0.0061775479564460944, P70=0.0014746875010456094, P71=0.0012311217904803359, P72=0.0034734709122362178, P73=0.007064196866941207, P74=0.021391998156032325, P75=0.04062742299016968, P76=0.026100651013212843, P77=0.017180006165016883, P78=0.022362138884341937, P79=0.016038211835304395, P80=0.008060953702337828, P81=0.006525989289191609, P82=0.005163000739859205, P83=0.014827437104397167, P84=0.02527206376875405, P85=0.02816117515082434, P86=0.06332092989887836, P87=0.06497840315784957, P88=0.01760254631796823, P89=0.0060061800016656605, P90=0.002686866307823063, P91=0.0017332562150870847, P92=0.0032051406703537282, P93=0.004536242234558276, P94=0.011365200580973034, P95=0.028759979037291147, P96=0.043619822449973554, P97=0.035703710805355206, P98=0.01660089285259353, P99=0.0022352513578379595, P100=0.0008408626525635153, P101=0.0056873592728020955, P102=0.008792356581733538, P103=0.0010863735422609686, P104=0.0029488836118437664, P105=0.012968457478718564, P106=0.010804651317320082, P107=0.0038425287218287803, P108=0.0007149107319680316, P109=0.0002821862912021693, P110=0.001861650887937314, P111=0.005970772363603841, P112=0.009898733356060688, P113=0.006978835624271011, P114=0.0031707883183466363, P115=0.0012102929397305713, P116=0.004049648510660873, P117=0.004377632790124938, P118=0.0015684423901323639, P119=0.002809840995678559, P120=0.0035306722745049803, P121=0.0004047698335749554, P122=0.001267932705547119, P123=0.0012409608752047816, P124=0.00039330197446308887, P125=0.00021948112997406525, P126=0.0004859357686446265, P127=0.00020230493494541999, P128=9.324445131317198e-05, P129=0.00011320455060659856, P130=5.186574523274633e-05, P131=5.268274884851986e-05, P132=3.0162659311135693e-05, P133=0.00010297798898727733, P134=3.3999213778463794e-05, P135=8.497154278340082e-05, P136=0.00010823924062867385, P137=5.717805116411645e-05, P138=0.00041551037475123906, P139=0.0005126094585007589, P140=0.0005497504913736165, P141=0.00022226037752308235, P142=8.214072763646208e-05, P143=0.00011631492398011015, P144=7.500018472860304e-05, P145=0.000127174051631208, P146=0.00010881303252467587, P147=0.00012151377847376482, P148=5.3551934968623976e-05, P149=8.172183254802356e-05, P150=3.651426258130031e-05, P151=3.1185980844372675e-05, P152=2.989994051861107e-05, P153=2.444901106588575e-05, P154=4.389364834813623e-05, P155=6.268966742671652e-05, P156=0.00011296169653286725, P157=0.00012834403280700905, P158=0.00011562058138329393, P159=0.00018503946283290963, P160=0.00029553075920190995, P161=0.00022340044240761898, P162=0.0002844992998715595, P163=0.0005508712867349931, P164=0.0005258537994018754, P165=0.00037306251059806536, P166=0.00018401564734843104, P167=9.096070814346234e-05, P168=6.057171291910813e-05, P169=4.887015104559062e-05, P170=7.54318276321835e-05, P171=7.919833923361865e-05, P172=9.309552888177539e-05, P173=9.870319468307554e-05, P174=0.00016574417987044393, P175=0.00012002567799316494, P176=7.180278501811681e-05, P177=8.065802717617797e-05, P178=2.924376165157245e-05, P179=2.6295244268439907e-06),Row( P0=-575.6443885739582, P1=29.08738493095333, P2=-31.356810829312593, P3=-7.458394920408812, P4=-18.62452133742414, P5=-16.51234040316789, P6=-8.919816612070457, P7=-3.5144472685415007, P8=-9.850098119044132, P9=-9.892097631631223, P10=-11.552462829286782, P11=-3.8838417000961143, P12=-6.9404525102341585, P13=-10.324120357949218, P14=-0.8934258925984592, P15=-5.237514274235066, P16=3.28841113955826, P17=0.6052527267872486, P18=9.11549590975225, P19=7.657011799267911, P20=5.03358172226696, P21=8.666939191689917, P22=2.449187233067371, P23=3.284837288325551, P24=1.12711722074675, P25=1.1558330572597586, P26=-1.4358254901507215, P27=-0.9781551280183024, P28=-1.977407920396487, P29=-0.4054336986934681, P30=-1.6507753230400275, P31=0.24911669982375798, P32=-3.467216621340196, P33=-1.5607008236225326, P34=-4.126729538857459, P35=0.8532938346889032, P36=-0.5118504776084257, P37=1.8727271201563391, P38=-1.1498109845507887, P39=-0.9116241615182998, P40=0.5958546429512827, P41=0.5852697520425446, P42=0.5644639604707652, P43=0.5546396552506405, P44=0.5878543873607281, P45=0.6516490337541537, P46=0.6794644045560118, P47=0.6832139763339053, P48=0.6835777145685076, P49=0.7202911746509617, P50=0.7102280502960651, P51=0.6568393078525626, P52=5.232092018596687e-07, P53=1.7052499794436646e-06, P54=2.692835711560992e-06, P55=2.6715951471092883e-06, P56=5.691148297005154e-06, P57=1.759452395397601e-05, P58=9.65099106304362e-06, P59=1.963480276548244e-05, P60=0.0002174852828887531, P61=0.0011231086850858013, P62=0.008480642748651531, P63=0.020669852782870403, P64=0.021562625502037896, P65=0.037036981482680684, P66=0.05660543760890313, P67=0.049847185420009904, P68=0.030317699992149772, P69=0.026544341251817222, P70=0.011387438761013224, P71=0.0019134387540376667, P72=0.0006262324332891836, P73=0.0015174450140558442, P74=0.0034339224860635213, P75=0.007599450144608197, P76=0.009072256778619024, P77=0.018277657727565698, P78=0.02989532929965134, P79=0.05619616337163691, P80=0.07885447997393262, P81=0.05782529187147676, P82=0.042010195605524776, P83=0.023779286792813156, P84=0.028704038920855716, P85=0.04788766731768864, P86=0.12883999030080395, P87=0.2787577465856862, P88=0.13868971154558557, P89=0.01624616773623833, P90=0.011158446982928906, P91=0.04405690745330657, P92=0.08492046267355825, P93=0.045528912975836285, P94=0.04902904081247167, P95=0.04293152342197827, P96=0.02177457745938146, P97=0.009527479986366414, P98=0.009567594049318387, P99=0.0177567346125671, P100=0.07868579591949514, P101=0.04833099748691457, P102=0.020603767764820785, P103=0.018077263040297258, P104=0.00609703277236985, P105=0.0021733563797003658, P106=0.006947759985690245, P107=0.034369742205294776, P108=0.010642134873918793, P109=0.0014210736103211492, P110=0.006593773434020949, P111=0.005904634764971207, P112=0.008991917769559215, P113=0.015276847225728689, P114=0.008959438805525773, P115=0.008453794346141746, P116=0.007515868132468708, P117=0.004197953510154778, P118=0.0027846621969691778, P119=0.0031002803891529273, P120=0.004875758961715759, P121=0.0031683159826697183, P122=0.002771509679646238, P123=0.005231348145354954, P124=0.0036320276821871582, P125=0.009157800991106649, P126=0.004794998253818486, P127=0.0012347922154251983, P128=0.0009541994066155437, P129=0.003108038849202874, P130=0.0008675732943023912, P131=0.0007871018896445491, P132=0.001402615791292617, P133=0.0012276088059209602, P134=0.0007337654479161953, P135=0.0007565691677125444, P136=0.0025896402649442742, P137=0.0015795577929106658, P138=0.0008371188353994373, P139=0.0010910347062143234, P140=0.0004056031275512133, P141=0.00045120938807922684, P142=0.00031397519265208474, P143=0.0004193732804429631, P144=0.00031430625901815126, P145=0.00032234916703002565, P146=0.00032218119268490384, P147=0.00037918599543410226, P148=0.0004064254861490731, P149=0.0006200478790519272, P150=0.0004896921697999953, P151=0.0003983502077541738, P152=0.00027559034594738704, P153=0.0004760173841649978, P154=0.0003098072998942684, P155=0.00014849374497615154, P156=0.00012443886117202946, P157=0.00013604904171279862, P158=0.00022083592319924433, P159=0.0003057164580764335, P160=0.00037140373582099796, P161=0.0003187106437968045, P162=0.0005373927725983245, P163=0.0003692678556436801, P164=0.00034882759838348644, P165=0.0002050042855441217, P166=0.00023040884720149825, P167=0.0001937199417535055, P168=0.00017169245185115115, P169=0.0001585759362912509, P170=0.00010878896212032375, P171=9.122061913789283e-05, P172=7.546592554929297e-05, P173=7.67286509445184e-05, P174=8.367190241306965e-05, P175=8.425903845458113e-05, P176=0.00015902490085854183, P177=0.00014783461411953266, P178=6.124112417971571e-05, P179=5.604677802702475e-06),Row( P0=-726.7476637254676, P1=42.760573959208834, P2=-4.908058907780596, P3=-3.2086521428795267, P4=-10.382104411311115, P5=-11.741378072308711, P6=-7.832704787388573, P7=-1.6153628979661228, P8=-3.818129093981078, P9=-4.01258781302189, P10=-7.233052092608469, P11=-3.174233448837988, P12=-5.843511984143476, P13=-6.188424266051722, P14=1.8072140857727337, P15=0.2183983506531325, P16=3.041432638218992, P17=1.2646963091445609, P18=7.361394910729364, P19=1.4398196517526887, P20=5.436853823110951, P21=7.002773779883904, P22=4.4903609328548395, P23=6.2609072278704145, P24=2.838892567210528, P25=3.0043018942452213, P26=-0.6024103625541269, P27=2.482840434805025, P28=-0.7787967418374877, P29=3.3639157987919193, P30=-2.5701807919547144, P31=-0.7441400758555116, P32=-3.252609266655801, P33=1.346156061219144, P34=-1.289729529096636, P35=-0.4002583774695615, P36=-4.886503126966044, P37=-2.3537559075969696, P38=-1.0514489915219616, P39=1.4775505850450072, P40=0.6740144144556942, P41=0.6347150933681414, P42=0.606232289969691, P43=0.6045677605928528, P44=0.6650113608226665, P45=0.6457326356300278, P46=0.6262401582723479, P47=0.6472011259331666, P48=0.6670245399804594, P49=0.6678734703810235, P50=0.6463715202636487, P51=0.6569923117696863, P52=9.54038587110308e-07, P53=3.3240494149592023e-06, P54=2.8064682633605378e-06, P55=2.2446522390498326e-06, P56=1.2184906521192367e-06, P57=6.900979029462625e-07, P58=1.024827510808154e-06, P59=2.7477212908098384e-06, P60=0.00023556315874836178, P61=0.002147916505025042, P62=0.004826640369295745, P63=0.009866132295774292, P64=0.014784485486404141, P65=0.010654434895078972, P66=0.00577631228796247, P67=0.010558425907509404, P68=0.005468798644695457, P69=0.001500041044171086, P70=0.00023303824520191896, P71=0.0004069067907572801, P72=0.0007435823841348835, P73=0.0005871205039746904, P74=0.001131772118534949, P75=0.0021515666061246944, P76=0.0015689133531617075, P77=0.002346907750968031, P78=0.0032201260776362904, P79=0.001411189160906583, P80=0.0005615667253572091, P81=0.0007400289255237577, P82=0.005163671061874855, P83=0.013934248460698124, P84=0.008268376648829753, P85=0.010673458456974315, P86=0.006907538333233403, P87=0.0013716408040961453, P88=0.00032849895125581954, P89=0.000348829820810666, P90=0.0006627611852878435, P91=0.0005980861117299087, P92=0.0002729523598824003, P93=4.630152327630941e-05, P94=0.0003662058507901581, P95=0.0018420198736636448, P96=0.0006251827073037118, P97=0.0002493687134802759, P98=0.0010072091209339625, P99=0.0019625005687070453, P100=0.00038708569596732653, P101=0.00021991083499101565, P102=0.0005238180460756182, P103=0.00018393396871921972, P104=9.139822904736318e-05, P105=0.0009911100648762456, P106=0.0011473055793311405, P107=0.0003449859919790855, P108=6.345913559735617e-05, P109=1.7018549102227722e-05, P110=7.360813516625336e-05, P111=0.0008026060956168512, P112=0.00043212099696520037, P113=6.528502794711928e-05, P114=2.9504632877430178e-05, P115=2.472981074592831e-05, P116=0.00011420838198928724, P117=5.260896356969311e-05, P118=1.5478712720384595e-05, P119=4.4104744667905826e-05, P120=8.224058358612409e-05, P121=3.4606777932030636e-05, P122=2.8048449961288633e-05, P123=6.572558193438262e-05, P124=0.00012327545627673775, P125=5.0780871965344606e-05, P126=2.748997366631208e-05, P127=1.2995853825513254e-05, P128=2.7048904378718794e-05, P129=6.403420906448158e-05, P130=6.112551631621545e-06, P131=1.4655200612421014e-05, P132=2.744255466733957e-05, P133=1.123810730849045e-05, P134=1.4946557821970934e-05, P135=2.3122441459233416e-05, P136=9.735598542400885e-06, P137=3.022205154839218e-05, P138=4.94949857749063e-05, P139=2.3245938407695112e-05, P140=3.969474935152894e-05, P141=2.1274370433325913e-05, P142=1.739443040621217e-05, P143=3.221545259856364e-05, P144=2.0837532475736366e-05, P145=2.4899193666383725e-05, P146=2.3338891295109138e-05, P147=2.7388712606995388e-05, P148=0.0001007027932300771, P149=7.03488092248672e-05, P150=7.564502009556388e-05, P151=2.0199650408012107e-05, P152=9.111692363878757e-06, P153=1.294814216853213e-05, P154=9.774814463439325e-06, P155=1.3619058320621354e-05, P156=2.8211982575882526e-05, P157=2.5629163576736295e-05, P158=7.784056342168768e-06, P159=8.707317403948312e-06, P160=1.5556844785756385e-05, P161=2.0474117049078828e-05, P162=1.4365613361811527e-05, P163=1.2992778811611766e-05, P164=3.867481732656505e-05, P165=4.521903788040705e-05, P166=5.157005150031513e-05, P167=1.5394752101218556e-05, P168=1.0775711776185105e-05, P169=8.774761546191921e-06, P170=1.2071832150475926e-05, P171=2.1778466918007375e-05, P172=1.0276180911968055e-05, P173=6.431663122930851e-06, P174=6.78328932119981e-06, P175=4.869447559169704e-06, P176=4.5882041658703765e-06, P177=4.517903219568383e-06, P178=2.485619597343241e-06, P179=4.2578197500298924e-07),Row( P0=-555.2961851081102, P1=22.495124138595845, P2=-32.47190798300708, P3=-15.79053700923235, P4=-16.68284542247469, P5=-25.00354176781704, P6=-13.338083748557532, P7=-8.715756308181204, P8=-10.284629240044309, P9=-9.880009845285764, P10=-9.151734016744209, P11=7.707310457099366, P12=2.776792286991113, P13=3.775457484846498, P14=9.29298314981879, P15=2.633248833548637, P16=1.0253043077041675, P17=1.4390455917692437, P18=3.260920535611269, P19=-1.074427745560183, P20=0.12644110933291353, P21=2.0950098678106035, P22=3.550400277107445, P23=5.104864784987455, P24=0.2247604236150794, P25=-2.0658559891655592, P26=-2.5544280919395086, P27=2.032661485881602, P28=-2.283742496418244, P29=1.4159989499879355, P30=-2.4956332014701634, P31=-0.15103280012658746, P32=-0.8460301774930641, P33=-2.7266975677806538, P34=-2.4020901862663986, P35=1.1703241931998138, P36=-1.9193780641130351, P37=-1.1062772110568824, P38=-1.707533520507782, P39=0.9978599815806459, P40=0.6032301055102208, P41=0.5873698555576539, P42=0.6187394352570508, P43=0.586218321264701, P44=0.5521077575790637, P45=0.6166479736673055, P46=0.6744827283430832, P47=0.6551961922849673, P48=0.6251696238119274, P49=0.5994459485647667, P50=0.5879826700272046, P51=0.5668157181527778, P52=9.643844722842317e-07, P53=4.1644477534614096e-06, P54=6.721440289172102e-06, P55=5.025092350410086e-06, P56=4.737815076357457e-06, P57=1.2529649277014245e-05, P58=1.1898280628246427e-05, P59=8.436120167283988e-06, P60=8.279434858150148e-06, P61=7.623354588867203e-05, P62=0.004675408039667744, P63=0.010988093776355361, P64=0.010617620238984266, P65=0.030935787707076605, P66=0.04662968299004294, P67=0.06832484614268099, P68=0.1848605521769932, P69=0.21763702574359178, P70=0.25249156552972607, P71=0.18283145638796053, P72=0.10858135935798618, P73=0.20645244491729087, P74=0.27173824741990976, P75=0.24512531838693255, P76=0.18737709104055134, P77=0.21903840036766947, P78=0.1418827494643307, P79=0.02857695548857781, P80=0.006783183926744735, P81=0.003764513218688947, P82=0.0035694418327666473, P83=0.016422522848022512, P84=0.06816040284397727, P85=0.15775465871367444, P86=0.13092279045594757, P87=0.1371768683302298, P88=0.24723171686163845, P89=0.1883202925242485, P90=0.07745339209588542, P91=0.034231224888059984, P92=0.046800658739218765, P93=0.06966550422630788, P94=0.3870827031503211, P95=0.302642315395415, P96=0.17960714235801475, P97=0.18575525300160084, P98=0.1834610061064668, P99=0.12689071284731757, P100=0.02787428626983899, P101=0.06885636524181363, P102=0.08943472298080728, P103=0.011892054748973048, P104=0.007923784500771377, P105=0.012093772964734104, P106=0.06542528557535447, P107=0.14473642197376446, P108=0.07633257357399711, P109=0.06995953664910479, P110=0.1115240047625566, P111=0.05881239744075854, P112=0.008648593651589423, P113=0.009293647029923465, P114=0.016679295278277807, P115=0.05191889216839103, P116=0.030763742924473692, P117=0.07283974911471629, P118=0.07513966147252428, P119=0.01115486677954743, P120=0.012146115973623185, P121=0.009751393692051366, P122=0.012863082855635198, P123=0.02949447749480238, P124=0.01806657172842354, P125=0.007043938932877209, P126=0.005136405087303901, P127=0.015059366016677785, P128=0.009282519012411748, P129=0.007444768681112415, P130=0.004901057282964929, P131=0.0036289045536609076, P132=0.003673298332540301, P133=0.001209712707516317, P134=0.0013408785935083872, P135=0.003385430540375937, P136=0.0034283340331854283, P137=0.0016273845457998259, P138=0.004477769990248448, P139=0.004262592373298226, P140=0.005169415032894101, P141=0.0036619316391320916, P142=0.0011509223443834254, P143=0.0008691564903185285, P144=0.0005231967801769654, P145=0.000478756260245504, P146=0.0008480026485277194, P147=0.0006197259359628862, P148=0.00035393438299644676, P149=0.0005295125934019321, P150=0.00024104145576513445, P151=0.00029102105824784294, P152=0.0003745229580307768, P153=0.0003782901030944574, P154=0.0004200739170704893, P155=0.00035113744140738694, P156=0.00040044843940029005, P157=0.00041234377558933393, P158=0.0005403225508039896, P159=0.000493905143185304, P160=0.0007135972947947198, P161=0.0004785670619738769, P162=0.0008235344081918084, P163=0.0007186565765334954, P164=0.0006016331089854463, P165=0.0005575897587214903, P166=0.00048601016816345646, P167=0.0003487098675883014, P168=0.00023104794846178655, P169=0.0002463595121923094, P170=0.0003232322089639236, P171=0.0004121685787411347, P172=0.00043688980922942824, P173=0.0003410046580782498, P174=0.0005007031984158376, P175=0.0005018325205653727, P176=0.0006259021775009011, P177=0.0007934713163462592, P178=0.00034795975122434164, P179=2.972449043041171e-05),Row( P0=-617.092596748125, P1=27.266623793067474, P2=-25.483132665930675, P3=-6.6481928685121465, P4=-19.704157398350358, P5=-21.009823250402334, P6=-7.902652441771173, P7=-3.2149380562463645, P8=-4.921401603877213, P9=-8.445135880715947, P10=-12.171830761820093, P11=-4.343919646906296, P12=-9.049028624577963, P13=-4.042912322050875, P14=3.2326425233374114, P15=3.7258096459987553, P16=10.701184838470247, P17=9.447228301204829, P18=14.562568086188465, P19=7.954042835120396, P20=4.068944508607022, P21=6.199945628532065, P22=-0.5604834033121049, P23=-1.6404308884951502, P24=-3.414475321272552, P25=-3.5055508851719335, P26=-2.0772835776993577, P27=1.8402362716983411, P28=-1.1543719588242496, P29=-0.25689847170251995, P30=-5.40273808399262, P31=1.2926600831802233, P32=-0.6843631441117424, P33=1.6930882019849358, P34=-3.6762947381312547, P35=-1.579223696470584, P36=-1.574697750212086, P37=3.7789636434338654, P38=1.9631908733272116, P39=-0.2379305360412824, P40=0.6585462123681824, P41=0.619129007685574, P42=0.5364654343610189, P43=0.5264844219428078, P44=0.5558918644600518, P45=0.5284589923358098, P46=0.496807513313926, P47=0.5452865089973725, P48=0.6710253398576503, P49=0.7328918618659787, P50=0.7174250541123222, P51=0.66247718841909, P52=6.082973290887097e-07, P53=2.1141956538757247e-06, P54=3.180847183946981e-06, P55=3.553737399016677e-06, P56=2.8492521975373337e-06, P57=1.7116958866808347e-06, P58=1.2673246270646117e-06, P59=1.63520207682583e-06, P60=8.238833087534497e-06, P61=8.339402571907899e-05, P62=0.0014591985867198158, P63=0.006346618921604818, P64=0.019113332815690123, P65=0.07724840339660129, P66=0.11586672040811499, P67=0.0822379466650356, P68=0.08715260988569216, P69=0.09091587569849736, P70=0.039456423261378444, P71=0.006028364198176038, P72=0.0004969852230999364, P73=0.00022591433767103224, P74=0.00022397822905293323, P75=0.000785952087804549, P76=0.0025374601903564763, P77=0.013433174866077773, P78=0.0514071996584016, P79=0.10045943394606761, P80=0.12182018804949601, P81=0.0843919252294077, P82=0.036322785266297564, P83=0.03166900126110472, P84=0.06725002899614253, P85=0.11614526503825698, P86=0.08437847455862946, P87=0.09540234525097538, P88=0.04330931965424749, P89=0.007063331519606255, P90=0.02864263231757758, P91=0.07610511111231082, P92=0.1156355512045477, P93=0.03119290628112996, P94=0.007522554852308202, P95=0.0013530824125821124, P96=0.0013211070994458632, P97=0.045799508476068965, P98=0.11914325904572841, P99=0.030340664057365838, P100=0.011164239039945368, P101=0.006359426697887974, P102=0.0020444540541787342, P103=0.00028005000542451024, P104=0.00020382967746561295, P105=0.0006209224191295829, P106=0.027871136574107988, P107=0.07274156271760396, P108=0.005173823655018485, P109=0.001082404765833297, P110=0.0015079697381339525, P111=0.0016164673142332697, P112=0.0015885205787810454, P113=0.00956162775604798, P114=0.017755698849540997, P115=0.0031789404389608636, P116=0.002771049319536571, P117=0.0029715414949710614, P118=0.0028140111054174187, P119=0.0029787701611322056, P120=0.004082132702420215, P121=0.001981077465350693, P122=0.0037067337128520784, P123=0.004151743996641643, P124=0.0014880981077380665, P125=0.0016888690717638184, P126=0.0026441755325493747, P127=0.0007956581422330337, P128=0.00025242542129270544, P129=0.0010265394276091995, P130=0.0005741664020088906, P131=0.0002535046991593313, P132=0.0006531898711519874, P133=0.0008934879449389982, P134=0.0006017227489769361, P135=0.0004851052260426496, P136=0.0005483383224288911, P137=0.000964677803816604, P138=0.0009623265120570926, P139=0.0003517274614966608, P140=0.00025331137981302775, P141=0.00016537585162280802, P142=0.00016251528462541234, P143=0.00017264143562456766, P144=0.0001121509093374003, P145=0.00012181282025816474, P146=0.00015453167158296357, P147=0.00022669229837498256, P148=0.0001452392158204691, P149=0.00028353431670602355, P150=0.0001794359220564995, P151=0.00012354954509852784, P152=6.676315584138511e-05, P153=5.2741883850004966e-05, P154=6.474739979161596e-05, P155=9.56045620210984e-05, P156=0.00012192621206960283, P157=0.00010828993844600953, P158=0.00014543674649293095, P159=0.00015652658157359005, P160=0.00020979256626532907, P161=0.00016158584709928382, P162=0.0001375024532059327, P163=0.00010424470284091696, P164=9.165083114622287e-05, P165=0.00010909890885300404, P166=0.00011770573137165886, P167=9.068467319293339e-05, P168=7.02566574050423e-05, P169=4.453179813689359e-05, P170=3.156992032942046e-05, P171=3.869333685129551e-05, P172=5.644371000822367e-05, P173=6.000055838491518e-05, P174=5.8371319730495545e-05, P175=3.226419472042111e-05, P176=5.123428985240227e-05, P177=5.9568466502193906e-05, P178=2.534676517672379e-05, P179=5.3753699473739944e-06),Row( P0=-569.8737583881175, P1=29.99892746709336, P2=-28.285095436520063, P3=-9.990198984786892, P4=-24.014337596121308, P5=-19.686798447170546, P6=-9.6507705838201, P7=-6.170873150794953, P8=-12.14240801197425, P9=-9.161383094895458, P10=-13.607192870813396, P11=-5.164323112901233, P12=-8.699735597539181, P13=-12.047853278102393, P14=-0.9568854984257703, P15=-3.3716177583041653, P16=6.245448121984058, P17=5.357520919755719, P18=12.191334357688275, P19=10.115890208161384, P20=5.622105567629507, P21=8.483490157496467, P22=0.10128412701015903, P23=2.462484219020363, P24=-0.059612566271457064, P25=1.486354408517163, P26=0.8604188210067792, P27=1.2213919411652856, P28=-2.330803324773617, P29=-1.7817885703159815, P30=-4.30209224554031, P31=-1.0151408091824863, P32=-1.3871050653577581, P33=0.38802417706798586, P34=-1.794007990010798, P35=-1.7095152272053444, P36=-0.8803951657002792, P37=1.8290054267405405, P38=0.413632669674732, P39=0.6117706259305232, P40=0.6096690605672157, P41=0.5395993231935333, P42=0.5336957723795589, P43=0.5707636835699241, P44=0.6258047627670525, P45=0.6265454127514036, P46=0.6267195331283075, P47=0.6051380466389534, P48=0.6608759361135479, P49=0.700717924346001, P50=0.6607019446808926, P51=0.6075229360899751, P52=4.060343745029656e-07, P53=1.483799319813839e-06, P54=2.5994679351436243e-06, P55=1.689705431478661e-06, P56=1.2480604464396186e-06, P57=1.390314529646772e-06, P58=1.6456597593487129e-06, P59=1.3957659548583576e-05, P60=0.00022523175988985628, P61=0.0020157541927448416, P62=0.010721202701406455, P63=0.020472097456837427, P64=0.03419617096043525, P65=0.0873304632960034, P66=0.11183017206674241, P67=0.08290496894718348, P68=0.03848604394932665, P69=0.01992378008725169, P70=0.027079055411569335, P71=0.010115136953301311, P72=0.0013587076459192513, P73=0.0014624233818589928, P74=0.001978723720654248, P75=0.0033315570547383377, P76=0.007127190956061569, P77=0.021416130664039884, P78=0.056779145940473154, P79=0.14432205447117435, P80=0.23466198089896673, P81=0.2317443920289939, P82=0.27644855856045536, P83=0.15646862365046962, P84=0.21135041891271233, P85=0.10553572589797292, P86=0.021050990494249334, P87=0.02966390417153092, P88=0.07644185211490562, P89=0.040007159100225016, P90=0.03972871155982217, P91=0.08181222799261678, P92=0.10569282019647927, P93=0.0776353195536493, P94=0.14802657781837603, P95=0.08086525195739534, P96=0.05877041851723093, P97=0.3264721152338715, P98=0.17311280473356358, P99=0.0975426010161224, P100=0.05700575822482968, P101=0.04000647495595121, P102=0.02681580165775021, P103=0.022792466939260046, P104=0.004794545556308082, P105=0.045766619570797536, P106=0.05466121725538913, P107=0.022465703092549458, P108=0.007846771443447639, P109=0.0014229537307112422, P110=0.00518562018058287, P111=0.0102149010466198, P112=0.01466257727733995, P113=0.011311359050313328, P114=0.0023595116798444784, P115=0.0045335375835760415, P116=0.01214608474879342, P117=0.01725578306769021, P118=0.004457237471951016, P119=0.0029750652415769076, P120=0.014477970257721372, P121=0.004353711424327272, P122=0.006428254222883264, P123=0.006752627082720145, P124=0.00391482283259095, P125=0.0031785783652690447, P126=0.003667158269757644, P127=0.003130636892359564, P128=0.00084374429661648, P129=0.00261905825123249, P130=0.00122821194768993, P131=0.0009428382365699043, P132=0.002420346153964636, P133=0.00100350591720816, P134=0.002282486346044563, P135=0.0030001717513308533, P136=0.0016423583281499066, P137=0.002802927764165542, P138=0.0022015864065960955, P139=0.0018047556911953398, P140=0.000579581316147526, P141=0.001000255144203738, P142=0.0005882766405636888, P143=0.0010472401945465376, P144=0.000961992968562804, P145=0.0008618742002199339, P146=0.0007006346482845463, P147=0.0003747887445121901, P148=0.0006091203292595231, P149=0.0005286596231436063, P150=0.000339119967826826, P151=0.0002632280298525503, P152=0.00016633358197281855, P153=0.0001055134250911885, P154=0.00019405507103818995, P155=0.0001977883689883303, P156=0.00019714101351501665, P157=0.00015974150525822366, P158=0.0002268172640448589, P159=0.00026556779577378804, P160=0.00034282287235063493, P161=0.0004202567480525706, P162=0.0004978732811090942, P163=0.00037228660600728124, P164=0.00048672173078015255, P165=0.0003733851442477616, P166=0.0002401349462666743, P167=0.00024949888422456863, P168=0.00019591082374931763, P169=0.00017114387092433947, P170=0.00019095145855534064, P171=0.00020491014019443586, P172=0.0001327309704757953, P173=0.000206236530720391, P174=0.00017547663869599872, P175=0.00021403547771115973, P176=0.00028445676632253943, P177=0.0001684663877268529, P178=0.00010147410688102672, P179=1.3500548250433445e-05),Row( P0=-674.7457550983736, P1=37.62858138214181, P2=-29.095321806963415, P3=-6.564904778952689, P4=-13.298850701033171, P5=-20.84094011646523, P6=-8.348708468844661, P7=-3.9189436752703237, P8=-5.089504979051889, P9=-8.164898417470573, P10=-10.004327870451732, P11=-0.41436440139084035, P12=-4.548381917109888, P13=-0.4259857112988227, P14=5.96592336816167, P15=3.204115141938536, P16=2.682555894145113, P17=0.18520479303134607, P18=6.9207534136548485, P19=4.694565117076244, P20=4.771485254613557, P21=7.536343809076902, P22=3.670833456249118, P23=0.7958770096637127, P24=-0.715234950982523, P25=1.4239355609991402, P26=0.475189928648098, P27=2.1905681004506414, P28=-3.2724822599947556, P29=1.820282919295573, P30=-1.5159903198707123, P31=0.9767796174871789, P32=-3.758156399705296, P33=-3.093698927837725, P34=-1.0700763283764652, P35=2.4682854282178117, P36=-1.886114371464863, P37=-3.8075290294853086, P38=-3.587225456505337, P39=1.0186822902533326, P40=0.6396927174704693, P41=0.5955639762343266, P42=0.6054294417916278, P43=0.6739962206715887, P44=0.630679101310671, P45=0.5903115512037461, P46=0.6380946682644665, P47=0.6801929386100546, P48=0.6710480111946583, P49=0.6746130816537914, P50=0.6991595532105036, P51=0.6827358424724177, P52=4.938087744997639e-07, P53=1.792249323048655e-06, P54=2.318859849536686e-06, P55=1.3079096922601291e-06, P56=5.929830959966535e-07, P57=7.43972925770253e-07, P58=9.797508832821303e-07, P59=1.2345895810267616e-06, P60=1.7322612512186815e-05, P61=0.00034933399985500886, P62=0.010635338913007225, P63=0.02563403051515805, P64=0.018338420972166987, P65=0.012387705388586847, P66=0.015366519940392334, P67=0.01875900811333415, P68=0.011525645578632282, P69=0.01513763620472852, P70=0.03938998171512465, P71=0.03742804406575958, P72=0.010566556751540505, P73=0.0031136836546184076, P74=0.004178049453127102, P75=0.007343174696794644, P76=0.008151085424573963, P77=0.01558825005423544, P78=0.01843848902429496, P79=0.012198710157621789, P80=0.009040421691983335, P81=0.008674797591433862, P82=0.007098148271957062, P83=0.013899668390493648, P84=0.026303888164573127, P85=0.010861238160114407, P86=0.004261282637580567, P87=0.012738086321615756, P88=0.027763159838892857, P89=0.03602682103777469, P90=0.03237615894655893, P91=0.00860807288123099, P92=0.003342732170243071, P93=0.003708348773536387, P94=0.00740627967239933, P95=0.013821977386721085, P96=0.01572904858808077, P97=0.007239057647656973, P98=0.0006056972555235525, P99=0.003536513218892291, P100=0.029440005643535026, P101=0.01098843738786658, P102=0.011970405847305798, P103=0.005403094683674462, P104=0.0008637039539858379, P105=0.001597459845439762, P106=0.0009697641242537547, P107=0.0007181489459878708, P108=0.002072560211522406, P109=0.008158503351023562, P110=0.00941703697728848, P111=0.012303324221643064, P112=0.0031834820834176726, P113=0.0007758118854147865, P114=0.0006097996619387808, P115=0.005671470192160157, P116=0.0072923293232924555, P117=0.0027780520450012437, P118=0.005093899596447118, P119=0.0016330472008728339, P120=0.0018412934467164836, P121=0.0022118599883825456, P122=0.002369388558241442, P123=0.003610288808527411, P124=0.0015106558304712721, P125=0.0003905293256151923, P126=0.000361336968058025, P127=0.0006391121683053274, P128=0.00042377704489688755, P129=7.885538533352335e-05, P130=0.00012365666955078866, P131=0.00013276907901474953, P132=0.0002503111253093038, P133=0.00017953159275338465, P134=0.00014323249352988886, P135=0.00014019617888278544, P136=0.0003041456611929384, P137=0.0005021883717555309, P138=0.0001259040316085859, P139=0.00019306751577869533, P140=9.106872720410049e-05, P141=6.0153112037986444e-05, P142=5.7781355916078234e-05, P143=6.0030165536943094e-05, P144=5.056111511018415e-05, P145=7.489383554953717e-05, P146=9.371965182653164e-05, P147=0.00015121843599227573, P148=6.744739812498114e-05, P149=8.076979183434389e-05, P150=3.225880245314903e-05, P151=2.0074289195982172e-05, P152=1.5992288029236406e-05, P153=2.182183157933127e-05, P154=3.0845541288131166e-05, P155=1.7589535943870938e-05, P156=2.1089936609260494e-05, P157=2.9727428947135832e-05, P158=2.3958194383267468e-05, P159=1.560931827440988e-05, P160=2.1876923689845516e-05, P161=3.995724315423437e-05, P162=6.356108526308904e-05, P163=4.47582416317249e-05, P164=5.248744209270606e-05, P165=6.988211049533567e-05, P166=4.0446013881750945e-05, P167=2.2162200024316855e-05, P168=1.7233025887141412e-05, P169=2.0999074981672347e-05, P170=2.3921346624912794e-05, P171=2.812598163036683e-05, P172=2.6171502983642096e-05, P173=1.6277456159704024e-05, P174=3.499220412229868e-05, P175=1.2733815764264096e-05, P176=1.3732373381434109e-05, P177=1.5383418467100698e-05, P178=7.252732561746678e-06, P179=7.671598850619962e-07),Row( P0=-603.2640880257211, P1=29.601982907832337, P2=-24.033081490098386, P3=-6.552002196178058, P4=-18.115538753422943, P5=-19.60485326891206, P6=-9.13513292245662, P7=-4.47031279081262, P8=-6.76523909493007, P9=-6.188797493421955, P10=-12.235579242072378, P11=-4.204536445280096, P12=-7.771196163074621, P13=-4.496219815688832, P14=3.7515801461174934, P15=4.02368099870298, P16=10.567856226354042, P17=10.01867883385428, P18=15.000759552969626, P19=9.987615007704933, P20=4.25884979972693, P21=4.541255431136, P22=-3.1082242542257887, P23=-2.7287716954593506, P24=-4.3747804349292325, P25=-2.664410103997519, P26=-2.7647614011360115, P27=1.8037513309926123, P28=-2.7567461398218676, P29=-1.8065301875371573, P30=-4.838257593730282, P31=0.6215616022179685, P32=-2.178788707041962, P33=0.6785530522369326, P34=-5.361324196403958, P35=-2.3596826339372465, P36=-0.8271853313653003, P37=3.259923110794434, P38=1.3418645338054809, P39=-0.270702277380053, P40=0.6569096719182201, P41=0.6073241987218092, P42=0.5786921463994141, P43=0.5536384943218661, P44=0.588450278587058, P45=0.5417400359053434, P46=0.5254366446126066, P47=0.5846967018855553, P48=0.6773535395729139, P49=0.7549979851989616, P50=0.7165940375179283, P51=0.653490786478164, P52=6.870415561455552e-07, P53=2.8100143229527107e-06, P54=5.763179190303032e-06, P55=8.393328359230947e-06, P56=1.570767487832079e-05, P57=1.1326467296597156e-05, P58=1.8155859468164564e-06, P59=1.5296262846004538e-06, P60=1.0780053112056782e-05, P61=0.00011381674971926343, P62=0.0017506448802487341, P63=0.006785663898402376, P64=0.01883734119989694, P65=0.09903055575543483, P66=0.15056017863825702, P67=0.06903866508058608, P68=0.06608480815504904, P69=0.07874826485840304, P70=0.07419282699208239, P71=0.019547580380948122, P72=0.0010076924491028343, P73=0.0003396013506266271, P74=0.0002528435246705217, P75=0.00042252113967465154, P76=0.001656521240477935, P77=0.009812440881191983, P78=0.0555547760000155, P79=0.1329849927675026, P80=0.1316814167730905, P81=0.10816397787404479, P82=0.06679936443968423, P83=0.04319260498352605, P84=0.12229883183193159, P85=0.1401042461408325, P86=0.042681700895743684, P87=0.08511634121138573, P88=0.15033949766942956, P89=0.02424274821128259, P90=0.02464020613435019, P91=0.11540964704689535, P92=0.14559709990525996, P93=0.0360052172518045, P94=0.024604568526624744, P95=0.002197507603051823, P96=0.011060100026830002, P97=0.1417189122873123, P98=0.06536162365537442, P99=0.01643649026663706, P100=0.017237207258471023, P101=0.011563514816435166, P102=0.0019902490348093172, P103=0.0004563749060162217, P104=0.00017954126339969688, P105=0.020715685057593015, P106=0.14360012936890026, P107=0.03702772563375996, P108=0.006058992756173724, P109=0.0017649032604181874, P110=0.001465349262191896, P111=0.0016848541393773243, P112=0.013822057735464428, P113=0.044724624855195426, P114=0.007176829928829495, P115=0.002616021023966658, P116=0.0036267878427680453, P117=0.00678874132720676, P118=0.006387165845111058, P119=0.0024701930567077755, P120=0.00239721429732725, P121=0.0030543007785903396, P122=0.0071534171089671795, P123=0.005701991679564998, P124=0.0018563089797492798, P125=0.00295830093163533, P126=0.0044102721977057075, P127=0.0010580707493210704, P128=0.0005584746748188459, P129=0.0006811306280400893, P130=0.001188176343194126, P131=0.0003989398374457993, P132=0.0002852982608896533, P133=0.00023288159944138625, P134=0.0006552266178012189, P135=0.0002418516655727118, P136=0.0004376839195621304, P137=0.0013523122291902972, P138=0.0003321720638701426, P139=0.00033961865779107717, P140=0.00037183119352478516, P141=0.0001676199348764374, P142=0.0001571000093334633, P143=0.00022904054023416268, P144=0.00015502604019877048, P145=0.00022370534770196063, P146=0.0002004075008695637, P147=0.00025310182947022725, P148=0.0002622461387345936, P149=0.0003210769484652559, P150=0.0002164723593527281, P151=0.00014111372473697814, P152=0.00010303345937975775, P153=0.00010337798637968108, P154=0.00014221071821921496, P155=0.0001956931378229149, P156=0.00023544515609857228, P157=0.0003171046964222433, P158=0.00033116084210994893, P159=0.0002650086876737364, P160=0.00031698886479112736, P161=0.0003088924062767177, P162=0.00028953878397553677, P163=0.00017232792846094545, P164=0.0003462848607292326, P165=0.00023369410456393805, P166=0.00013739028612573447, P167=0.00013045072591762812, P168=9.366146601494443e-05, P169=5.7395382016938863e-05, P170=5.1748425032575575e-05, P171=6.920532268380891e-05, P172=9.065698172710989e-05, P173=8.485710830195168e-05, P174=0.00011788325655974301, P175=4.936613465995955e-05, P176=7.112373040778345e-05, P177=7.042474326899206e-05, P178=2.8813070368592826e-05, P179=3.086512977883897e-06),Row( P0=-602.2693259540252, P1=27.261800303450894, P2=-27.6760804190773, P3=-11.13929331406802, P4=-17.598867284285106, P5=-20.600357538800505, P6=-8.634218177732524, P7=-2.150846557652831, P8=-4.293751924727937, P9=-9.756480412579139, P10=-13.186590907031777, P11=-2.0687112837814516, P12=-8.360746970159402, P13=-5.178746483121529, P14=5.183812648795557, P15=4.664424203046483, P16=10.165018819655414, P17=10.39400975006291, P18=15.807166488727548, P19=9.18618568686289, P20=4.506118360281295, P21=5.871604994903771, P22=-0.011062235325129613, P23=-2.1409624068474904, P24=-2.531248707062819, P25=-4.079083271630748, P26=-3.396565296762622, P27=0.6355921735173845, P28=-1.9929671005370924, P29=0.39150589325493557, P30=-4.84352212072238, P31=-0.35250427210960755, P32=-1.9024072636639733, P33=1.618815435160471, P34=-2.5605001020464067, P35=0.09372602901538046, P36=-1.1746849348186827, P37=1.4602226828347271, P38=0.9509004394833639, P39=0.8521626563545791, P40=0.6824433291913372, P41=0.6046243802981885, P42=0.563437836438867, P43=0.57260068137535, P44=0.5532766398176838, P45=0.5567268727237319, P46=0.5472556915417801, P47=0.5712719479546436, P48=0.6683159371849431, P49=0.7671574660746029, P50=0.777848400654924, P51=0.7090119128912094, P52=2.3963300786376022e-06, P53=8.733420246126056e-06, P54=5.7984166401528e-06, P55=2.963860192312645e-06, P56=3.153269640019423e-06, P57=4.478786523980031e-06, P58=2.82680619060523e-06, P59=2.0424961638517137e-06, P60=9.070879411917461e-06, P61=0.00010957933524868698, P62=0.002148953525370631, P63=0.009924860397748864, P64=0.02587115377318068, P65=0.08537032509874999, P66=0.133953233436645, P67=0.1027247219417433, P68=0.06328076040399741, P69=0.038257056307710965, P70=0.020560048161363677, P71=0.004035065266769549, P72=0.0005515919075933864, P73=0.0004588306693215587, P74=0.0005299221627461856, P75=0.0013162076206486204, P76=0.004229483828626019, P77=0.02568903372007333, P78=0.05102784917613185, P79=0.08098971324614401, P80=0.08979411027394701, P81=0.09902271507696228, P82=0.09144009068206967, P83=0.09830389116546218, P84=0.14593197760227686, P85=0.10276254470164695, P86=0.05185182082906801, P87=0.05237884679314761, P88=0.024006046859225, P89=0.037833062361655706, P90=0.04850945601099662, P91=0.04701109368553723, P92=0.05568927396760828, P93=0.025152228574133042, P94=0.021967071916770158, P95=0.0066953078084057835, P96=0.0177792085333543, P97=0.11918877295244076, P98=0.1354029140478236, P99=0.1546662260519056, P100=0.05919210152598284, P101=0.013273680124041787, P102=0.0016175951328787081, P103=0.000375655976640394, P104=0.0003056167989667401, P105=0.004757429695311475, P106=0.014374124892124545, P107=0.013406017752548993, P108=0.006394417702111323, P109=0.002832741330219496, P110=0.004699852927873113, P111=0.00972054284510527, P112=0.004322201374646925, P113=0.004047926499889912, P114=0.002877723225272495, P115=0.005350055233918617, P116=0.0030348853131257608, P117=0.003720756123731364, P118=0.005454869837172167, P119=0.002532863715930297, P120=0.004600242664199069, P121=0.0029510155736503663, P122=0.00716384573854006, P123=0.0033775869777404955, P124=0.002364151740755752, P125=0.002550917608443895, P126=0.004110050277158635, P127=0.0025230796222634275, P128=0.0006220148303429301, P129=0.0007197999944024044, P130=0.0005472033237307475, P131=0.00027351663108749575, P132=0.00027562440963854783, P133=0.00039384983053950706, P134=0.0004735152138765846, P135=0.00030640587392569466, P136=0.000696695775575737, P137=0.000742064203050225, P138=0.0012004951097411157, P139=0.0011486038142230369, P140=0.0002063254935314207, P141=0.00015735372162635128, P142=0.0001257868717278231, P143=0.00015617827327040506, P144=0.00022519611206895247, P145=0.00022337945054972584, P146=0.0002500859556624537, P147=0.00020882608548079175, P148=0.00018815805450413039, P149=0.00012849875913434913, P150=6.863151165504182e-05, P151=6.152396610913405e-05, P152=5.3869223313852153e-05, P153=8.042987528176166e-05, P154=7.849638887357305e-05, P155=9.918750813612465e-05, P156=0.0001393181491225591, P157=0.00024648372638028793, P158=0.00036991427137972115, P159=0.0004048593779598203, P160=0.0005699037764454541, P161=0.00033788893562956357, P162=0.0003380357344675331, P163=0.0002817016245530865, P164=0.00036626102387761375, P165=0.0003198248342732751, P166=0.00019914386774026044, P167=0.0002575890544056607, P168=0.00018832390082100192, P169=0.0001533026024592904, P170=0.00019122530941823778, P171=0.00033325043614545075, P172=0.00033793342707354857, P173=0.0004335844272830712, P174=0.00022030294200847464, P175=0.00018975775185842034, P176=0.0002636898633494047, P177=0.00034998635780552614, P178=0.00019003661727920828, P179=2.0335566847219053e-05),Row( P0=-512.4512460943963, P1=5.005125350957107, P2=-33.060073366793425, P3=-12.761384834296447, P4=-25.44636997690877, P5=-24.837755744091556, P6=-12.760632842125064, P7=-6.153828264947294, P8=-10.55386768534916, P9=-8.144988406518655, P10=-5.327930740700227, P11=9.83280384120617, P12=7.593813058678911, P13=15.554143291812796, P14=15.690365387594833, P15=2.553282671871723, P16=-1.221241901534164, P17=-10.580795678157715, P18=-0.9129349449549031, P19=1.9752433124042608, P20=-2.56995564416689, P21=1.4741162419264326, P22=-7.778094453070431, P23=-0.4351092828475151, P24=3.637818204206827, P25=5.8517107455728485, P26=0.5103223333260865, P27=-1.2991843816207833, P28=-3.939207838235132, P29=2.3343585491713252, P30=-1.8362352283510708, P31=0.21308983220480393, P32=-4.052529122013234, P33=3.505233377384692, P34=1.3915202765298258, P35=1.716686852631749, P36=-2.96172130248664, P37=1.8346055130852188, P38=0.9161110646192202, P39=0.8076310055048336, P40=0.6583074371158685, P41=0.6351760565673513, P42=0.6298960172415532, P43=0.6774446535443073, P44=0.7358944474553969, P45=0.7403989316155721, P46=0.6743452292619568, P47=0.5661386408030528, P48=0.5460666370694348, P49=0.5544152713950093, P50=0.598705401326275, P51=0.6336853011133188, P52=1.0722995172659191e-06, P53=3.936520737874472e-06, P54=4.930771684655147e-06, P55=5.999057492467159e-06, P56=9.915626766941964e-06, P57=7.525022303484017e-06, P58=5.05574714877287e-06, P59=6.664700937304223e-06, P60=6.008081602913966e-06, P61=6.827789716419076e-06, P62=1.805525865577377e-05, P63=2.487308621128296e-05, P64=2.9247514721395846e-05, P65=0.0003307888451122129, P66=0.005405856527735727, P67=0.0220301389297871, P68=0.0507083272335888, P69=0.08524943646464282, P70=0.18302001798758188, P71=0.3351755189956971, P72=0.4713889843252644, P73=0.7615645229791863, P74=0.5966662835090248, P75=0.33118672727595716, P76=0.09041164581717379, P77=0.012971433738784783, P78=0.004331857726743152, P79=0.0019816936916657707, P80=0.0010320082787707932, P81=0.000854617666131412, P82=0.000894752980272497, P83=0.001637669054250592, P84=0.0027023960082578425, P85=0.003667745717177254, P86=0.005282406067094092, P87=0.024121399922032983, P88=0.18251767756333198, P89=0.4915887629455429, P90=0.8658600049050839, P91=1.7220676625338105, P92=1.8969041786792002, P93=0.8307092706627452, P94=0.35615310535081024, P95=0.16811001246360746, P96=0.028190781716448062, P97=0.0008601315204279367, P98=0.00033574539109234994, P99=0.0005657970851722264, P100=0.0012152882320195064, P101=0.013317530896104534, P102=0.036735364796776325, P103=0.22855850950116843, P104=0.20998176985356443, P105=0.34128839332162997, P106=0.324923202916768, P107=0.1272985618989683, P108=0.028799182335995654, P109=0.0002547861351493911, P110=0.0030328140449643534, P111=0.03287372918867586, P112=0.15261636829131384, P113=0.06721684882136865, P114=0.011020773560415706, P115=0.016494606383921206, P116=0.008894988476181863, P117=0.036440106499642515, P118=0.1230628338362884, P119=0.12513971450038633, P120=0.02316086571968151, P121=0.015363460037657053, P122=0.047896233566559386, P123=0.053574757917499445, P124=0.07467275819977098, P125=0.03413932054477298, P126=0.032151531685033034, P127=0.04779464856097429, P128=0.02651392106533055, P129=0.046202970772900545, P130=0.006085156128202434, P131=0.010339071655205418, P132=0.014020113583783458, P133=0.04266096754506861, P134=0.01223275920885615, P135=0.013617256445937191, P136=0.02826411470160859, P137=0.02274693433879843, P138=0.006589402851540476, P139=0.024217467771675213, P140=0.007800107325482039, P141=0.0008539676998639137, P142=0.0011368567847694325, P143=0.0023810864454357314, P144=0.0014477313646648108, P145=0.003289899714428942, P146=0.004442986494061518, P147=0.0029186144505748608, P148=0.006624696731861936, P149=0.0035388514774646633, P150=0.0052523570000463015, P151=0.0017823060860015288, P152=0.0013995058453420042, P153=0.0007489187074956585, P154=0.0001923031584282112, P155=0.00026334403159338654, P156=0.0003021106389946249, P157=0.0004852473039009327, P158=0.000817373763440401, P159=0.0007165240513707437, P160=0.00104763570560428, P161=0.0014635600602441313, P162=0.001957646999264326, P163=0.0015942406777859131, P164=0.0012042932849757805, P165=0.0007609945009061481, P166=0.0005881793030251693, P167=0.000645344428081142, P168=0.0005673369425155681, P169=0.00040220626878941705, P170=0.00046831268377792043, P171=0.00041262751661725255, P172=0.0002914188702455702, P173=0.0003570604714641201, P174=0.0005781293735532956, P175=0.0007033954922806231, P176=0.0008307859910189285, P177=0.0006700048609841816, P178=0.0003620663874513527, P179=4.4300621420673904e-05),Row( P0=-536.8756943062665, P1=11.312782825521149, P2=-35.649412339239696, P3=-13.821620243406795, P4=-24.66785536826211, P5=-19.63999035313239, P6=-13.426538584975347, P7=-4.914302858044172, P8=-6.705238051278334, P9=-0.3061862884599106, P10=1.7798205512805547, P11=15.745308725992038, P12=9.485374229055138, P13=8.53606048533778, P14=6.882053434683538, P15=-4.472938280576091, P16=-4.47044631966212, P17=-6.752348172880994, P18=1.8702709132435715, P19=3.803971111344899, P20=-3.785973169708044, P21=-0.0626529792305606, P22=-2.3913582469660484, P23=2.9510737237966493, P24=4.115529546892349, P25=0.6628296546191976, P26=-4.006838762127103, P27=0.5881114945939208, P28=-1.337378618506763, P29=4.071779648550707, P30=-2.3196924600153506, P31=2.3420249290661066, P32=-1.5297190045074804, P33=1.9752783481740344, P34=-1.009161397173695, P35=1.154880265758205, P36=-2.813594324552312, P37=-1.9933842883131978, P38=-1.9861401482242926, P39=-0.05680503845618041, P40=0.6040452590924368, P41=0.555812892048576, P42=0.5313560183879462, P43=0.5763586699890869, P44=0.5992092556566949, P45=0.6706584480147935, P46=0.7079582181176876, P47=0.7159642629893419, P48=0.655223371891381, P49=0.5747505436457856, P50=0.5559442919705965, P51=0.5972940502614389, P52=1.9198138761041835e-06, P53=6.662745881538007e-06, P54=7.823712109610802e-06, P55=8.427326557849247e-06, P56=9.90863214616981e-06, P57=7.861258658624546e-06, P58=4.016863295002063e-06, P59=5.367871814652338e-06, P60=1.2062838501514367e-05, P61=1.5695623964076362e-05, P62=2.0670989232060552e-05, P63=2.3366597097244864e-05, P64=2.1506896281213737e-05, P65=5.107887698452463e-05, P66=0.0002061162465174369, P67=0.0015394365659439524, P68=0.02452871147927218, P69=0.06510191818991613, P70=0.07824889274715308, P71=0.09790150787399639, P72=0.22255280796218682, P73=0.4218842279890332, P74=0.45370999006814255, P75=0.7924359607246447, P76=0.8117346274862377, P77=0.43911924313037826, P78=0.15232555815843757, P79=0.10438106445311268, P80=0.0903693669471041, P81=0.1294713559711729, P82=0.15005027924966935, P83=0.07924725622928279, P84=0.02316789817177917, P85=0.004489272503438854, P86=0.0019016276150956785, P87=0.0016065339521284508, P88=0.005917457850820571, P89=0.08143077742637032, P90=0.164870471781084, P91=0.18113581911268514, P92=0.5547654702411874, P93=0.7861030278278733, P94=0.29297887859599225, P95=0.26865912936387165, P96=0.30013607099814277, P97=0.682481687990726, P98=0.1585465423811741, P99=0.05277928600465554, P100=0.04584433397594023, P101=0.05367862415053784, P102=0.12551794224889742, P103=0.26011097443077, P104=0.14855775896029308, P105=0.16339814067542702, P106=0.09190260620809723, P107=0.05889570365239839, P108=0.03314958783965378, P109=0.2813371430038009, P110=0.18214852790089558, P111=0.04789524419014446, P112=0.02576715201823738, P113=0.027543125081661287, P114=0.028449901971481648, P115=0.017422966949039987, P116=0.023663729343928075, P117=0.04254836424545019, P118=0.0965837605387794, P119=0.047091480611057945, P120=0.00973790860646673, P121=0.08164075697893515, P122=0.027483328373865232, P123=0.021924546853431242, P124=0.022263680171962036, P125=0.054537772993338784, P126=0.03811627035281718, P127=0.012308320277574879, P128=0.004746799115984842, P129=0.005116679975057567, P130=0.005228685923449172, P131=0.01095447593549411, P132=0.005070148113917483, P133=0.002041902457092405, P134=0.002874586306073711, P135=0.003791579830504857, P136=0.00641549673612002, P137=0.0020940349452921265, P138=0.00333015119792853, P139=0.01762868294555124, P140=0.004049462052458247, P141=0.0007168086146576706, P142=0.0011382352501776239, P143=0.0023584431905482946, P144=0.0010428509446569947, P145=0.0014944160872670399, P146=0.0029964796432259628, P147=0.0017635682856518172, P148=0.00229192114547749, P149=0.001731959974135333, P150=0.0006195453014896501, P151=0.000594324359932715, P152=0.00034908977461061184, P153=0.00023788933375427896, P154=0.000742549560464001, P155=0.0004544014397073885, P156=0.0004909152739929846, P157=0.0006933757271187257, P158=0.0008989588954349488, P159=0.0009052010775747489, P160=0.0011960200926744208, P161=0.0011075813749774962, P162=0.000990097910698555, P163=0.0006341757334322824, P164=0.0005774829305182477, P165=0.000653222409903071, P166=0.0007800189498638939, P167=0.0007753250493917522, P168=0.0005218226070348805, P169=0.00019024588673459444, P170=0.00011242705509642326, P171=8.58579344582946e-05, P172=0.00010252641764622854, P173=0.00012033288679375212, P174=0.00020643747752286412, P175=0.00018613656784441257, P176=0.00016895792361376437, P177=0.00024044106078619735, P178=0.00017016157745883894, P179=2.152437539826135e-05),Row( P0=-702.0595080972643, P1=45.0926116644568, P2=-11.615326998253657, P3=-2.6043800238532833, P4=-9.439283733990356, P5=-12.72376070450779, P6=-10.201836480326914, P7=-0.1250388957790661, P8=-0.224497953058858, P9=-7.531550008002799, P10=-9.93816950020045, P11=-0.48641378558942605, P12=-6.649184611364929, P13=-6.738199387354973, P14=3.2811113623299533, P15=1.112643124115145, P16=0.6315321266806893, P17=1.8115886106900962, P18=7.8354092425587645, P19=3.1655333647166617, P20=4.444565229479697, P21=6.863301372509861, P22=3.3489270864435574, P23=0.6499177240904336, P24=-0.028702701523246825, P25=0.2937479375879268, P26=-0.39748011512108683, P27=2.6589683680537424, P28=-0.7339862723635369, P29=1.474185365040169, P30=-3.091264083459259, P31=-2.0754154157194784, P32=-2.3642175890016617, P33=0.20026563855241494, P34=-2.8424823821841696, P35=-1.597644470651642, P36=-4.130583319155184, P37=-0.9925316994476046, P38=1.0129172940098163, P39=-0.09358690827979667, P40=0.644030084632114, P41=0.622671757801452, P42=0.5576212508303416, P43=0.6046139271612547, P44=0.6195476420650791, P45=0.5968724707549371, P46=0.6118147839684805, P47=0.629885213587829, P48=0.6858759998461879, P49=0.7337542524658467, P50=0.7182311921037327, P51=0.6484072368066904, P52=4.312471213589976e-07, P53=1.6236068918051354e-06, P54=3.9360975155854335e-06, P55=6.455558705863216e-06, P56=9.630963520924026e-06, P57=9.438476246285313e-06, P58=3.613815028157425e-06, P59=1.6065325330825406e-05, P60=0.00046119589277615684, P61=0.0015032029089165483, P62=0.004772678048367121, P63=0.008807624218191002, P64=0.014101537012867291, P65=0.013875786650708108, P66=0.0054614222331316, P67=0.0032671250575789777, P68=0.003915993151036614, P69=0.0010402879057426241, P70=0.0004767178207545737, P71=0.00047892822374962033, P72=0.0003779399246258087, P73=0.0004387567735361639, P74=0.001055257618859687, P75=0.0009283893730229666, P76=0.0009383971192053446, P77=0.004898218376867659, P78=0.012028528441306655, P79=0.009737193274196622, P80=0.0038589716678734164, P81=0.0014367035127987617, P82=0.0008145681194368412, P83=0.0012514775241495996, P84=0.004324279662104729, P85=0.004920878756623839, P86=0.001349628461101901, P87=0.0005482889919822362, P88=0.00031991127393671444, P89=0.000974467141036243, P90=0.004094397905722362, P91=0.004896480532014251, P92=0.0028109151310950764, P93=0.0009907432893629564, P94=0.0016648809211918276, P95=0.0007036816528336696, P96=0.00015045748391953834, P97=0.00011747035475579948, P98=0.00367954864044129, P99=0.023239145107845707, P100=0.005407936663844752, P101=0.00039401655239024787, P102=0.00014802497194750739, P103=2.8172525857705576e-05, P104=3.0121187929152324e-05, P105=0.002017632671423917, P106=0.010084823378907584, P107=0.00396329056556891, P108=0.00011676773955447719, P109=8.374670728492153e-06, P110=0.0003276030382115472, P111=0.005933735083044515, P112=0.007399736831164183, P113=0.0008228126812421663, P114=6.058572588082996e-05, P115=0.0001777226596822782, P116=0.0006663983449722283, P117=0.00032751592948989446, P118=9.980110260761647e-05, P119=0.00017572095478460872, P120=0.0003445648405447217, P121=0.00016485383507081205, P122=0.00021319070937605774, P123=0.0001869847884618467, P124=5.9616618577175355e-05, P125=6.341155257356074e-05, P126=4.356968695820558e-05, P127=3.677183913210666e-05, P128=1.0963783967112981e-05, P129=5.9502437140067155e-06, P130=1.2372950461460072e-05, P131=5.1391487808392555e-06, P132=1.0667099094847831e-05, P133=1.1881522808026668e-05, P134=8.482240548589598e-06, P135=1.1381009273134811e-05, P136=1.5282919174137093e-05, P137=4.087293228347285e-05, P138=5.6705509462420764e-05, P139=4.736995082301769e-05, P140=2.6372496086410267e-05, P141=1.158516265923977e-05, P142=9.594394801362594e-06, P143=1.764900129019135e-05, P144=1.6404131271953564e-05, P145=2.9182778695656238e-05, P146=3.816822968632712e-05, P147=3.275130875265057e-05, P148=2.0080791875706837e-05, P149=3.1255435539548435e-05, P150=2.5535876352389218e-05, P151=3.351373253830687e-05, P152=2.488902354315799e-05, P153=5.499039741753362e-05, P154=5.1288755144813516e-05, P155=8.44909041917177e-05, P156=0.00012966298512678758, P157=0.00019227289590821933, P158=0.0003206212736594866, P159=0.0003315616856739643, P160=0.00029555427878540514, P161=0.0002944985030255861, P162=0.0002635875137782185, P163=0.00016854691241891817, P164=0.00014029362114246853, P165=0.00011398763904284428, P166=9.377486615341117e-05, P167=5.159714404249554e-05, P168=2.9127323407125628e-05, P169=2.573260253819994e-05, P170=3.355372672054935e-05, P171=3.752662157982262e-05, P172=5.767648425067491e-05, P173=3.00681086495451e-05, P174=2.9099715580099573e-05, P175=2.1963561699399595e-05, P176=3.418131686144495e-05, P177=2.1920112687145765e-05, P178=1.1652730192694473e-05, P179=1.2891415516228362e-06),Row( P0=-520.231724826655, P1=2.591566628521742, P2=-35.69792226644244, P3=-12.76666565801457, P4=-24.677712394384045, P5=-23.181322760187502, P6=-13.06661284226681, P7=-4.912658701899921, P8=-6.4450766320209505, P9=1.5907362800169462, P10=3.9850470472769244, P11=18.373997230661082, P12=7.969863995110754, P13=6.714626710288276, P14=1.754839927174667, P15=-10.861632873337584, P16=-5.158375171007349, P17=-3.4188487378471697, P18=4.465017570982387, P19=0.6314726881794792, P20=-7.432753467717205, P21=1.3515827627926735, P22=0.8876192482042904, P23=3.516995740500697, P24=-0.1418391988277587, P25=-1.7450631606656555, P26=-4.146741514915276, P27=2.370391779802248, P28=0.14929473929510634, P29=1.0060330978810192, P30=-2.3669195984984674, P31=2.1930543331863, P32=-3.9073337009335014, P33=3.0614211085300624, P34=-1.6474702716914726, P35=1.2899953160928135, P36=-3.1437784681632497, P37=-0.09768541335535556, P38=0.5152496516566987, P39=0.6615466547910146, P40=0.6093123205263201, P41=0.5825162898267054, P42=0.5433146681243614, P43=0.593953456569566, P44=0.6171183389432175, P45=0.6418450694340863, P46=0.6694899103847755, P47=0.634193879996925, P48=0.6890752494466957, P49=0.6568911436320961, P50=0.6150924817967424, P51=0.6189807004039708, P52=1.0090202448567276e-06, P53=3.0233662448671767e-06, P54=6.866904427805091e-06, P55=1.3701330837282975e-05, P56=2.322382623641109e-05, P57=2.2897960591000248e-05, P58=1.5444959526382906e-05, P59=1.4103335235175586e-05, P60=1.427718394501188e-05, P61=1.580813121186437e-05, P62=2.206314702425777e-05, P63=2.167650082912663e-05, P64=1.214484161689764e-05, P65=3.052278898628968e-05, P66=0.00012293512734570604, P67=0.0007068058539960044, P68=0.007277581402179245, P69=0.026265043842496328, P70=0.05206379401032637, P71=0.07310104078540791, P72=0.14053089707209265, P73=0.2734827348460264, P74=0.4358979560301462, P75=0.4072435023132319, P76=0.5123541512875402, P77=1.4036499037999524, P78=1.2349270697329668, P79=0.6120839087770762, P80=0.2095504765095973, P81=0.20551338976400066, P82=0.2359100602862709, P83=0.13107308723079428, P84=0.04527137774870544, P85=0.004228297289792851, P86=0.0006898760964783548, P87=0.0011724864656411647, P88=0.0019857599934459125, P89=0.005445695164459032, P90=0.0529818850521666, P91=0.18598706704188028, P92=0.5501132085261731, P93=0.6926251171520917, P94=1.5985056981144936, P95=1.283722659982028, P96=0.5389280875793835, P97=0.7931207900662128, P98=0.9592313829387262, P99=0.2792703298141034, P100=0.0132678698032157, P101=0.0015494941506068287, P102=0.001606192734639003, P103=0.01353064468683413, P104=0.020731453289408792, P105=0.06708052213570745, P106=0.1087627512589275, P107=0.13543992195154259, P108=0.06124783084040253, P109=0.04091348706350005, P110=0.28513115465159655, P111=0.9162825413994344, P112=0.1265881366017316, P113=0.0464483109238837, P114=0.1253663944734974, P115=0.07040085300872799, P116=0.011193307092204084, P117=0.010365173205838593, P118=0.02289221371587349, P119=0.05129388472305104, P120=0.0505670922418428, P121=0.016696817401235943, P122=0.029398294754818338, P123=0.13270294706511934, P124=0.02973699691120804, P125=0.02755075280182776, P126=0.0508685599747498, P127=0.030741222473567982, P128=0.04323279721076488, P129=0.010286158296350704, P130=0.00304875594669618, P131=0.004932466797801161, P132=0.010360458369957545, P133=0.03192858739791596, P134=0.019581221117402434, P135=0.003713875440048193, P136=0.012511403685737691, P137=0.027758875986630693, P138=0.01077432357188909, P139=0.0024704460074349544, P140=0.0019311037807041177, P141=0.0027730893252080435, P142=0.0005776641133173047, P143=0.0010154186854057705, P144=0.004106280630519779, P145=0.0036249915679617294, P146=0.00235390903825229, P147=0.002291206280808842, P148=0.0017962543899707913, P149=0.0020452195254881136, P150=0.00137536583335397, P151=0.0008975142322340285, P152=0.0009377119256345926, P153=0.0005994403984057803, P154=0.0006312553140678217, P155=0.0008141677724320848, P156=0.0012018698789146352, P157=0.0014177062199046686, P158=0.002260165349763755, P159=0.0031096878900611982, P160=0.0030776483489426123, P161=0.0031667179119383142, P162=0.003056046777292079, P163=0.0017616667555346898, P164=0.0013188980771783336, P165=0.0008492954827649624, P166=0.0011184812551190582, P167=0.0010798780494709762, P168=0.0008247144792026781, P169=0.0006207664894309531, P170=0.00040922087686125794, P171=0.00027556969242782957, P172=0.00028150809245620093, P173=0.0003609835821390641, P174=0.0004929494086786328, P175=0.0007153682305656021, P176=0.0008794163333880861, P177=0.0010049697713326647, P178=0.00038390902100593266, P179=3.5343949425625375e-05),Row( P0=-708.1797396933613, P1=52.493470755294716, P2=-21.627656733451666, P3=-3.305487372436121, P4=-9.664312637290415, P5=-10.89391233936866, P6=-11.730043479266273, P7=-2.6432889113668017, P8=-1.0209604996798125, P9=-7.025854902972595, P10=-10.718209761597056, P11=-1.1594581013433949, P12=-4.711985257982945, P13=-7.389110390280818, P14=2.6451784942861063, P15=2.165342435377948, P16=3.210252775465928, P17=5.341708657966724, P18=11.790106388230658, P19=5.718109358222429, P20=6.703005520080569, P21=4.441803113278267, P22=1.1218609796606387, P23=1.4047695010722316, P24=1.5021795719836564, P25=3.5324160646429865, P26=-0.3846853908874137, P27=1.7439212365454286, P28=-2.0984016772581797, P29=1.3109659000760212, P30=-2.3289258491864575, P31=-0.820415190774617, P32=-2.4272157612479193, P33=-0.7226084289024447, P34=-2.245545633288621, P35=0.170903032905715, P36=-3.2285093016765605, P37=0.007162615933509642, P38=-1.4320070976063972, P39=1.9826385953960024, P40=0.5851331115879821, P41=0.52075132105752, P42=0.5455351870337617, P43=0.5953825604177897, P44=0.6454755945734614, P45=0.6331626700703691, P46=0.6354730129742576, P47=0.6423351521051255, P48=0.7010244603539537, P49=0.7350688315430645, P50=0.7040174154498557, P51=0.6639467291059253, P52=5.258553366994784e-07, P53=1.751356921463508e-06, P54=2.606786216172194e-06, P55=2.954106731648675e-06, P56=3.0996258566491946e-06, P57=2.60379484092027e-06, P58=1.0857172511130518e-06, P59=1.8203161168326378e-06, P60=0.00013291759293733988, P61=0.0011505339513701832, P62=0.0028482174127421878, P63=0.006118804160245454, P64=0.01148583574963265, P65=0.015054928987223373, P66=0.011419662069718327, P67=0.00733265858347178, P68=0.002937654118869252, P69=0.000413634040592867, P70=0.00016064729112095827, P71=0.000254363614828969, P72=0.0004670336122536071, P73=0.0004229494463379135, P74=0.0005252694060434049, P75=0.0014080958517278157, P76=0.001270190444020367, P77=0.0035935174269167394, P78=0.007228951491998224, P79=0.0068809608284601115, P80=0.005989687786171255, P81=0.003934740871061044, P82=0.0023269433041761874, P83=0.003249731092297116, P84=0.005606888343639628, P85=0.0024817272547890154, P86=0.0006814192971459285, P87=0.0006611934771046401, P88=0.0003626018625919709, P89=0.0001961199281099238, P90=0.00022379857817936462, P91=0.0019067277270716535, P92=0.005545142468036403, P93=0.004571021280610346, P94=0.002469564106250554, P95=0.001479439795507881, P96=0.0009283157304616648, P97=0.0002149194218232246, P98=7.604599610057248e-05, P99=0.0001228956707833227, P100=0.002436580474220019, P101=0.0029728793798476263, P102=0.0027877718689592913, P103=0.0007176974870749133, P104=0.00010011534309399625, P105=5.3099781929152376e-05, P106=7.098003166822569e-05, P107=0.0009500783184807222, P108=0.0007666205587655584, P109=0.0002524880874627275, P110=0.00010413190598995104, P111=6.473367160131729e-05, P112=0.00036930581170247564, P113=0.0009683657976859253, P114=0.0004413791534849, P115=0.00025315034868600775, P116=0.00022213621282183543, P117=0.00032450055877660813, P118=0.0002814056213061629, P119=6.331548124929427e-05, P120=9.012177739155885e-05, P121=8.124848727457383e-05, P122=0.00010979271398134866, P123=0.000117761596378457, P124=7.379661046380673e-05, P125=5.833196664307841e-05, P126=6.191743996046703e-05, P127=2.532867233894753e-05, P128=9.924744972312907e-06, P129=1.0097133078680303e-05, P130=7.883106625837277e-06, P131=1.315098914099154e-05, P132=1.0179522930175843e-05, P133=8.8163799835283e-06, P134=9.151639558301158e-06, P135=9.644271650738126e-06, P136=2.762197870429482e-05, P137=3.477170746330144e-05, P138=8.693656201873599e-05, P139=6.973333642626787e-05, P140=3.834729885347396e-05, P141=2.250166999338977e-05, P142=1.840079899019244e-05, P143=1.549346186568398e-05, P144=1.3168117257172054e-05, P145=1.6125074080664043e-05, P146=1.311569443542556e-05, P147=1.7210824332748894e-05, P148=1.2333717831729423e-05, P149=1.1264124139935862e-05, P150=9.693284764105512e-06, P151=7.4035263353956074e-06, P152=1.0391536797427246e-05, P153=1.1696954627351746e-05, P154=3.1575470310653434e-05, P155=4.763783639775251e-05, P156=7.580773435456952e-05, P157=6.0120592374248395e-05, P158=9.549139545597917e-05, P159=9.167365251466287e-05, P160=0.00014632109729378955, P161=0.00020573403888989066, P162=0.00017126942809009618, P163=0.00015455297564761084, P164=9.183772095150278e-05, P165=8.936420227908887e-05, P166=5.45816298835445e-05, P167=2.8005288179602574e-05, P168=2.1868063281344853e-05, P169=1.5047373577009549e-05, P170=1.7919710828890597e-05, P171=2.226868576395714e-05, P172=1.9933997661182668e-05, P173=8.985859362057892e-06, P174=5.9517317747311355e-06, P175=8.719883185466654e-06, P176=8.855199652290207e-06, P177=1.1467070125044555e-05, P178=5.697528148961296e-06, P179=5.706409524612599e-07),Row( P0=-652.0343003427397, P1=49.44020158867855, P2=-23.266215027528496, P3=-6.62491722149555, P4=-12.988728440742028, P5=-19.43441101947508, P6=-11.202088080431334, P7=-5.017452802577276, P8=-6.395687265492209, P9=-9.126607808713532, P10=-10.989468887287781, P11=-3.4243402629602677, P12=-7.518686139783635, P13=-6.1777304196439875, P14=1.4892736702866765, P15=-0.3901811946410389, P16=4.500228481531977, P17=2.940279506295841, P18=9.557444580929165, P19=6.327312148172767, P20=7.238056423078936, P21=8.901804883066951, P22=2.724197213664003, P23=2.7906748989165275, P24=-0.12757846954940272, P25=0.05990641621882452, P26=-0.12253102740057056, P27=-0.03848595478873511, P28=-4.165591341295587, P29=0.4162229562824813, P30=-4.309899814006938, P31=-0.6384157891164544, P32=-3.65969498083373, P33=-1.4625937929787336, P34=-4.7022903879984135, P35=-0.17442464689213363, P36=-1.7110659441084888, P37=0.5397463750722543, P38=-0.5070172792327133, P39=-0.6990526834346559, P40=0.6292900374406074, P41=0.5719329723757542, P42=0.594503548589944, P43=0.5784464449391323, P44=0.5776944294013595, P45=0.6115334161551468, P46=0.6214454784212459, P47=0.6247675244758394, P48=0.6883594025703486, P49=0.7274087116819508, P50=0.692283660975476, P51=0.6487372500265051, P52=5.059323042954329e-07, P53=1.9224418717652616e-06, P54=3.3978473743995336e-06, P55=5.048380158056929e-06, P56=6.352621286831727e-06, P57=5.0588322018157715e-06, P58=3.2604563916914854e-06, P59=4.859615211560935e-06, P60=8.836415214938773e-05, P61=0.0013317353013098047, P62=0.011277801267956482, P63=0.01942596476429257, P64=0.022976521787482752, P65=0.036863715887354584, P66=0.028474528202013093, P67=0.01923389224186639, P68=0.01037712380857858, P69=0.015976739844233926, P70=0.02655861789684986, P71=0.009293394851972305, P72=0.001292090124012893, P73=0.00248602217087397, P74=0.00544867348670964, P75=0.008092863029371289, P76=0.009888057430663379, P77=0.019675354049764084, P78=0.036273044672558735, P79=0.04129508875122756, P80=0.0258284611610437, P81=0.008666470830312759, P82=0.00931947280390713, P83=0.020195701437784583, P84=0.024861166042564865, P85=0.02346946679371557, P86=0.021783259760837486, P87=0.018380903725590283, P88=0.047089789350515526, P89=0.0348924493133129, P90=0.021740236604932052, P91=0.014314919117999331, P92=0.015970364488468624, P93=0.0074421216074846905, P94=0.008708071231680065, P95=0.011853017732731747, P96=0.0015574036142550233, P97=0.0009056633065842894, P98=0.0009269588673665077, P99=0.004888885301777933, P100=0.008411652993308805, P101=0.006620632393769488, P102=0.006906298941669825, P103=0.001659906344018846, P104=0.0008620879311930621, P105=0.0016547086606015313, P106=0.008073705331761119, P107=0.010211361107373374, P108=0.0020986886296836655, P109=0.0007810900472895584, P110=0.0013917144711224864, P111=0.001860403615315598, P112=0.005965621338721989, P113=0.002781309318464422, P114=0.0028061578245004537, P115=0.0014552591807435215, P116=0.0022942905798751688, P117=0.0010091683159087702, P118=0.0006029453850918717, P119=0.0004955014568553077, P120=0.0021549996005718105, P121=0.0009302852558978155, P122=0.0007192123597446178, P123=0.0012435362237001718, P124=0.0005288043104288141, P125=0.0004076216232596522, P126=0.0006218765989707905, P127=0.0005637010735882409, P128=0.0001401920523282174, P129=0.00010100278918021097, P130=0.0003282748839862189, P131=0.00031302144677268577, P132=5.943899116075798e-05, P133=7.166766902098647e-05, P134=9.982703855853616e-05, P135=0.00010351115816814828, P136=9.09042820806311e-05, P137=0.00010754848129843713, P138=0.00040958779400745564, P139=9.853050380371288e-05, P140=0.000137850113456473, P141=6.88814586745319e-05, P142=3.7552258615920246e-05, P143=8.490947693202627e-05, P144=4.8013320329175575e-05, P145=5.914926852914603e-05, P146=8.42480504976068e-05, P147=6.473474897132648e-05, P148=4.729300468411693e-05, P149=6.088470065702405e-05, P150=0.00010704380349405379, P151=5.477501628713827e-05, P152=3.917289203825972e-05, P153=4.776434814055987e-05, P154=5.636768315053733e-05, P155=5.007585618578835e-05, P156=5.7402867051813224e-05, P157=4.01740024007523e-05, P158=3.540136279111562e-05, P159=6.043703329973652e-05, P160=0.00010156118944700803, P161=4.895262689427307e-05, P162=4.4809371438055685e-05, P163=4.2299522882398644e-05, P164=6.321406619386722e-05, P165=6.299139481501752e-05, P166=6.033018790174897e-05, P167=3.246770210018129e-05, P168=1.4175005842488637e-05, P169=9.116911254692117e-06, P170=7.819805837542636e-06, P171=1.3897852019223624e-05, P172=1.4144963196747668e-05, P173=1.8491966309700716e-05, P174=2.367106150710696e-05, P175=1.8447026802873988e-05, P176=2.292370226661969e-05, P177=2.9321090909589348e-05, P178=1.6388793030731123e-05, P179=2.0280265470198317e-06),Row( P0=-704.1938808251012, P1=45.82294534071122, P2=-11.412531506106005, P3=-0.44750195816890725, P4=-10.35189300217378, P5=-12.708106596974972, P6=-6.535197210992466, P7=-1.7739576916925022, P8=-2.3485734317231497, P9=-5.292205702686811, P10=-9.98276989416371, P11=-3.4192473151545175, P12=-3.7639603275905515, P13=-4.189070380408633, P14=4.520763879531169, P15=1.6948282955258434, P16=1.6603681203525855, P17=-1.3979662336956424, P18=4.03858026861296, P19=-0.6492698710372912, P20=2.498382793818694, P21=4.369537940548536, P22=2.1533765577525243, P23=2.3743646706126973, P24=0.5070892210790245, P25=2.2087234195479017, P26=1.6433424714343945, P27=5.194281746569911, P28=-0.3896180578338683, P29=1.6417188300239653, P30=-2.976760941442746, P31=-1.0701781886791357, P32=-1.8930237633057316, P33=-0.7088783069110309, P34=-2.0855753330670113, P35=-1.333744535795797, P36=-3.605881590897744, P37=-1.1336201386981828, P38=-0.5957333415706545, P39=-0.31858542692179476, P40=0.633879930392306, P41=0.5925150866531831, P42=0.6091167727383691, P43=0.64240537165061, P44=0.6829683501986712, P45=0.6574705925387417, P46=0.628273737747984, P47=0.6112651678056138, P48=0.6693580641883272, P49=0.6933890398538411, P50=0.6419272986047472, P51=0.6234785504593829, P52=5.976927588311821e-07, P53=2.4616914341558288e-06, P54=3.7023219516935483e-06, P55=4.918685296514046e-06, P56=8.636271195738338e-06, P57=6.481327290766527e-06, P58=3.1364071119549706e-06, P59=5.2108708196802315e-06, P60=0.00036434366252001047, P61=0.003506880927517219, P62=0.008529744731023631, P63=0.009895668579449214, P64=0.011257006547719993, P65=0.007652322506344992, P66=0.0019169982774850445, P67=0.0015021534517807834, P68=0.0031383470999402532, P69=0.012236358028297253, P70=0.015271231249164149, P71=0.006889498017534696, P72=0.0020138836676335944, P73=0.0012813614406536214, P74=0.000983682313415327, P75=0.0007123262917625556, P76=0.0009792941499931652, P77=0.001975001338385633, P78=0.0022729588699385074, P79=0.0014342015019175544, P80=0.0003476394404945195, P81=0.0002026026018245186, P82=0.0006140458291929831, P83=0.0013071645417846386, P84=0.001190234504232223, P85=0.0008079353155206099, P86=0.002780508508061961, P87=0.016890351568957553, P88=0.027432968531165083, P89=0.00832130990935109, P90=0.0037613096802203193, P91=0.0010136908242146172, P92=0.0003002011829909909, P93=0.00013292190312650022, P94=0.0007967352431865571, P95=0.0002483350737401238, P96=1.4854197013518438e-05, P97=6.591098108378173e-05, P98=0.00031519912447074903, P99=0.0016191976484998146, P100=0.0007415441392738999, P101=0.00021028330437708988, P102=7.844151687209718e-05, P103=7.111018224538124e-05, P104=0.00010689660003010771, P105=0.0002521134450089446, P106=0.0019047511923400411, P107=0.0010197963217603626, P108=6.221022583605369e-05, P109=1.9474664516187282e-05, P110=4.733718965437862e-05, P111=0.0002453405079671566, P112=0.00034421431140674286, P113=9.72725909456803e-05, P114=1.9913227642077162e-05, P115=6.29293742951955e-05, P116=6.958635013594352e-05, P117=5.636267505319699e-05, P118=3.893838288329191e-05, P119=0.00010696496639231531, P120=2.8210393748419313e-05, P121=3.7573092226585094e-05, P122=9.901743355264946e-05, P123=0.0001943987896714152, P124=3.073153167601907e-05, P125=3.239733423281708e-05, P126=0.00010939336492354792, P127=3.0683159203574156e-05, P128=1.706613506687918e-05, P129=1.67720946930175e-05, P130=0.00015925335550527505, P131=2.115312040279843e-05, P132=1.4767510401434985e-05, P133=6.792694278318437e-05, P134=4.986650430449474e-05, P135=3.688658199280029e-05, P136=4.5334226567851343e-05, P137=8.196473207170307e-05, P138=3.841218816774021e-05, P139=4.5953288788426934e-05, P140=5.5629413403583324e-05, P141=2.2817354685172612e-05, P142=2.4466552029334417e-05, P143=1.306121698917937e-05, P144=1.4462305069208168e-05, P145=1.5318520138080293e-05, P146=1.4911458246009174e-05, P147=1.731618658324938e-05, P148=1.301199057648428e-05, P149=1.70333278509903e-05, P150=1.2025761734378913e-05, P151=5.1868401796591485e-06, P152=5.686614095242762e-06, P153=8.067481351343508e-06, P154=1.1680974362352691e-05, P155=1.2945765095257947e-05, P156=1.0350286142818794e-05, P157=1.1761105371080384e-05, P158=1.1569944085951192e-05, P159=1.3167218028684102e-05, P160=1.2322702497707371e-05, P161=1.935333912561987e-05, P162=2.2937603181697587e-05, P163=2.0370781198954873e-05, P164=4.633197425563895e-05, P165=4.7513211561322996e-05, P166=3.1313994074854133e-05, P167=1.1206138251977621e-05, P168=1.0512936970673568e-05, P169=1.0517680147873444e-05, P170=5.602226215744644e-06, P171=1.4791016508439585e-05, P172=8.59805585831651e-06, P173=6.1482646182211995e-06, P174=9.313470496793122e-06, P175=1.3537532532306339e-05, P176=1.2466844137291812e-05, P177=6.790907682352095e-06, P178=3.7409842447996646e-06, P179=4.4846473761340084e-07),Row( P0=-600.9282827303766, P1=26.87803756709657, P2=-26.568236941848998, P3=-8.932790441526569, P4=-20.304049002162774, P5=-15.602869029602916, P6=-10.453247688688284, P7=-3.07705916317513, P8=-12.722222628494126, P9=-8.433682832739093, P10=-12.738219181081323, P11=-3.8807536063785175, P12=-10.419362683736255, P13=-6.672581288983028, P14=1.914949443233453, P15=-0.5463775691046685, P16=10.470936657565382, P17=6.001166880592776, P18=11.788801819513042, P19=8.319725165644094, P20=3.5382911968120165, P21=6.993343604940365, P22=3.2077121671808504, P23=3.0794606166459793, P24=1.2150984351531686, P25=0.0690684718599549, P26=-2.531134683698938, P27=0.850347425964859, P28=-0.9884635149695564, P29=1.1066281968917389, P30=-2.9259613203681174, P31=0.41939620378858083, P32=-1.6388884777539443, P33=1.1007318996050064, P34=-0.4204949035564607, P35=-1.2595808186755424, P36=-1.034145593736524, P37=-0.47613056359527084, P38=-0.1885588131402847, P39=0.2164627871303852, P40=0.6802949202807641, P41=0.5983905772982511, P42=0.5803553576349997, P43=0.5930371827509108, P44=0.5753380565774361, P45=0.5863177210298331, P46=0.6126042284937495, P47=0.6313219379283892, P48=0.6377486145588006, P49=0.6732247591838317, P50=0.6615828792013602, P51=0.6560876633209795, P52=5.60845539079163e-07, P53=1.981918033193254e-06, P54=2.4780597660084373e-06, P55=7.901336413221962e-07, P56=2.7507002203718614e-07, P57=7.247922022740678e-07, P58=8.160671815718338e-07, P59=1.8455909946800308e-05, P60=0.000333865435668605, P61=0.0014724656640674857, P62=0.005665370382060272, P63=0.014081968438601683, P64=0.019121758948394817, P65=0.05570624239189604, P66=0.09823840283387507, P67=0.1129942008864432, P68=0.05686354131189729, P69=0.034474727458401425, P70=0.024314607954617668, P71=0.004682538395354908, P72=0.0011582878750458524, P73=0.0013293057313196945, P74=0.0015325561538092965, P75=0.0032979564725863996, P76=0.008317835820537172, P77=0.021175235747246522, P78=0.04108212236626934, P79=0.06129080850048014, P80=0.08865296104226077, P81=0.08034990606021243, P82=0.09793488997276584, P83=0.1921702668289608, P84=0.214499220858638, P85=0.24419202592762138, P86=0.1401518750990975, P87=0.08798123870753409, P88=0.054820718531287393, P89=0.03383109143710389, P90=0.026196610789054862, P91=0.03376612947343566, P92=0.039084612094545175, P93=0.018410233406700355, P94=0.021845294817719976, P95=0.06810334925413716, P96=0.09013007330875604, P97=0.05266173989064229, P98=0.06221024703035857, P99=0.08117330821511604, P100=0.07898622582536739, P101=0.011266031439317165, P102=0.005137252975383705, P103=0.009731571026316463, P104=0.037158560584892114, P105=0.03922623979017323, P106=0.027844935200866245, P107=0.025968015676871303, P108=0.01508071022577492, P109=0.005254121400192775, P110=0.00679019567583907, P111=0.017555663814459967, P112=0.006574441679481364, P113=0.00244285518930207, P114=0.0010208285344483286, P115=0.00414393888530191, P116=0.010040288249793961, P117=0.002783367088864784, P118=0.0007361741164636065, P119=0.0011817957485038728, P120=0.0016285602576957578, P121=0.0013336367953340942, P122=0.0019544509112674473, P123=0.004150308954797096, P124=0.0017163118110907125, P125=0.0015081624388509351, P126=0.0023627240465419037, P127=0.013703899633230206, P128=0.003437830365375404, P129=0.004124306299878334, P130=0.0019232904424655382, P131=0.0011363025415660464, P132=0.002011047375064285, P133=0.0018083009677918212, P134=0.002431055209606471, P135=0.002012856280453387, P136=0.0031114122798167184, P137=0.003244003955659635, P138=0.0011359127455849896, P139=0.0007573523627106406, P140=0.0005067058446302985, P141=0.00020337668378823742, P142=0.0003184720713477524, P143=0.00025371243842052234, P144=0.00029832067706125854, P145=0.0003467386755820612, P146=0.0003595288145869125, P147=0.00033869411543658705, P148=0.0004697744009875262, P149=0.0004080394567718411, P150=0.00043449510906269927, P151=0.00026661991044257924, P152=0.00029820857123095815, P153=0.00017065625487281547, P154=0.0001057019574706236, P155=0.00012203366262632804, P156=0.00015786536614007264, P157=0.00015552176991554388, P158=0.00025134968989022887, P159=0.0002829590098208675, P160=0.0004671547604323952, P161=0.0005344717097642176, P162=0.0007981175095395527, P163=0.0012346924165655621, P164=0.0010167748334551617, P165=0.0010225607350248787, P166=0.000690519220373152, P167=0.0005235474533075017, P168=0.00039577780643637535, P169=0.0002815584744787435, P170=0.00021933223879151933, P171=0.00017093163220659454, P172=0.00019383008303081294, P173=0.00020999887335016742, P174=0.0001694379559113858, P175=0.00011900794649671113, P176=0.00012079459617828253, P177=0.00010398878344452468, P178=6.046190318859059e-05, P179=4.501364508587514e-06),Row( P0=-550.9503652246708, P1=43.00218341222113, P2=-24.764928855092652, P3=-3.0283108607814633, P4=-11.048492518980867, P5=-11.381547831464799, P6=-2.6559877575631665, P7=1.4661836951505445, P8=0.9480280598049559, P9=-15.826260108752416, P10=0.13806209282139648, P11=-3.2862105833224176, P12=-14.736297670740118, P13=1.9623201388584566, P14=-1.6486487467145312, P15=5.216303074320296, P16=5.800819778938541, P17=4.852973584229647, P18=10.216426659742115, P19=6.048273700627927, P20=4.621311528328076, P21=3.699806563290759, P22=3.5134031935298653, P23=-0.16481953948884626, P24=-0.8669290283324627, P25=-2.300732725821515, P26=-5.122884950873504, P27=-0.39813415366718335, P28=-2.080982149102507, P29=-1.2599453198422694, P30=1.173072141343234, P31=-1.0617885967453855, P32=-2.6809881936579623, P33=1.4729926453708735, P34=-3.5052754329124953, P35=-1.3043816571786468, P36=-4.416067349850694, P37=-1.092965847761881, P38=-1.7294335519858557, P39=-1.0002329740316451, P40=0.7492139907696576, P41=0.6895556700383108, P42=0.5897475225382619, P43=0.561303978532838, P44=0.5640277095948175, P45=0.5209330419941675, P46=0.5129250394459497, P47=0.551764960757631, P48=0.6167253139712261, P49=0.6502444918021932, P50=0.641662906171014, P51=0.6305881363397653, P52=6.661367018359018e-06, P53=2.026431025354279e-05, P54=7.028856613511528e-05, P55=8.28086930559388e-05, P56=7.050443501702737e-05, P57=7.162897740120511e-05, P58=3.8579823933863944e-05, P59=3.100900039395295e-05, P60=0.00024685383475028203, P61=0.0023707383226466193, P62=0.012609336359454217, P63=0.03940737469415009, P64=0.07149755071574744, P65=0.17845086518809114, P66=0.23336185600559525, P67=0.2502613629581249, P68=0.2514061720253501, P69=0.1310397292897282, P70=0.09756173155701087, P71=0.03721037928423733, P72=0.0024811969460297907, P73=0.0018279281053028332, P74=0.0020729692794375747, P75=0.002805365448690966, P76=0.0038412138918259213, P77=0.013092814352690496, P78=0.02850008233807234, P79=0.04412891342576577, P80=0.06307179040372794, P81=0.046524569193161204, P82=0.03795035916063793, P83=0.11423621746227777, P84=0.40218045333730895, P85=0.31461755703553784, P86=0.0787954942010791, P87=0.04109465895707367, P88=0.08192201601963309, P89=0.05542721196959449, P90=0.02305820140774426, P91=0.03265907312659859, P92=0.050839353418692466, P93=0.04100811238971201, P94=0.03423912958617141, P95=0.048520917697887175, P96=0.04907673106630438, P97=0.07973857250264603, P98=0.14919199183116716, P99=0.03131541779586364, P100=0.001950468193003005, P101=0.02592797039333663, P102=0.028868610459530902, P103=0.012900152875490603, P104=0.024946947943434366, P105=0.03528537760037867, P106=0.04618558688919227, P107=0.04088168038296929, P108=0.00919835894743298, P109=0.006867726926653397, P110=0.006683410424108385, P111=0.007415683814427384, P112=0.009064777099174465, P113=0.0035974564172405327, P114=0.0035017844061060706, P115=0.002567639272975427, P116=0.0014515577879315733, P117=0.0012539032656444585, P118=0.0021369680767672696, P119=0.0017274637415619608, P120=0.0009663138579388591, P121=0.0026906976677925163, P122=0.00482795129409724, P123=0.0029007301389746697, P124=0.0027631539513553603, P125=0.0018927490532701784, P126=0.003405441566380919, P127=0.0049812104574752, P128=0.002167437895296481, P129=0.0009169584893829403, P130=0.0011376215953716891, P131=0.0009427321598885562, P132=0.0011665403369016854, P133=0.0010888622709016697, P134=0.0006744461618027932, P135=0.0008650087503823416, P136=0.0004309126271904989, P137=0.0006695586516867894, P138=0.00033909247949020555, P139=0.00017907314369518642, P140=5.652067024705771e-05, P141=6.36061778214399e-05, P142=6.654517013593386e-05, P143=0.00012145934775943596, P144=0.00016055644253104654, P145=0.0003507806951359916, P146=0.0006079571782865724, P147=0.0008339836261774933, P148=0.0007138190919337358, P149=0.0008049805305856335, P150=0.0008971970390737935, P151=0.0010080156242907921, P152=0.0015703965857419872, P153=0.0019063086819924627, P154=0.0018570756426370804, P155=0.0023710379450490623, P156=0.0016490554760948237, P157=0.0006834407445086715, P158=0.0004970014575272503, P159=0.00026149803845896944, P160=0.0005509002365104395, P161=0.0010720026293833656, P162=0.001321955437892601, P163=0.0022266681597945294, P164=0.0011492198330815939, P165=0.0010387317218920188, P166=0.0007371715411572273, P167=0.0004377277819008052, P168=0.0018021646764201965, P169=0.0033619952119233864, P170=0.004416185334684222, P171=0.00348502698280914, P172=0.0029659334189567946, P173=0.0015449745669243486, P174=0.0007573039515195457, P175=0.0006575424562575877, P176=0.000497526601747728, P177=0.00039758075758140784, P178=0.0002668858563854042, P179=2.541400816658918e-05),Row( P0=-600.212924265456, P1=63.93143125499017, P2=-11.890372932956257, P3=-5.117580978756358, P4=-8.450876940728026, P5=-9.815274168398327, P6=-2.1120999053949894, P7=2.887816683290396, P8=4.284354223434086, P9=-9.961116131667474, P10=-4.186310385997256, P11=-0.7042447680793619, P12=-10.561453182677086, P13=-5.002789756703856, P14=-5.320416578524439, P15=-3.522663303541942, P16=-0.30833824649263064, P17=-5.1122726272389905, P18=2.160059633657165, P19=4.897935273987601, P20=5.059101958047353, P21=8.55047998704226, P22=7.277108352403568, P23=5.949663404487974, P24=7.226568213879449, P25=4.880120663474324, P26=-0.34412505681282457, P27=1.2485474005106527, P28=-1.7842418006856782, P29=-4.675443252338227, P30=-1.993533353615009, P31=-1.466534769099804, P32=-3.8593220569140385, P33=-2.5425584379113837, P34=-5.0200476400559015, P35=-2.3857686835421887, P36=-4.745759791274114, P37=-3.1508900133405287, P38=-3.6297725734184385, P39=-1.8053356225715935, P40=0.5766109590782406, P41=0.5474494555803908, P42=0.5302221888969327, P43=0.5442163252884248, P44=0.6180662207617075, P45=0.636065370532045, P46=0.6241424375736535, P47=0.6900943190430993, P48=0.6505352617791781, P49=0.5876792869850997, P50=0.5556847686456925, P51=0.5534045565452742, P52=5.2577899228064825e-06, P53=1.45586142613077e-05, P54=4.595476055061834e-05, P55=6.517935724973554e-05, P56=6.994717950817283e-05, P57=7.623043019655634e-05, P58=3.299064205260378e-05, P59=2.6395433279753266e-05, P60=0.0014449096383323736, P61=0.02140426241788276, P62=0.08851152237219173, P63=0.13954836184238398, P64=0.1235057975130945, P65=0.12371270142622122, P66=0.057383793136390894, P67=0.01585785609367895, P68=0.004213911545271149, P69=0.0012067068973668394, P70=0.0008944921004370581, P71=0.0021262952422459493, P72=0.012801886461133633, P73=0.019439635939040013, P74=0.044542910232278715, P75=0.056259388074815814, P76=0.03723376614846933, P77=0.12297218231734187, P78=0.20676538794683458, P79=0.15020182553444378, P80=0.1437585159156954, P81=0.07334394095613424, P82=0.03635608917797143, P83=0.026950813711118803, P84=0.032580525075815214, P85=0.02513005380561787, P86=0.023802222794556495, P87=0.0168236095253068, P88=0.014359057763403975, P89=0.018262827741569083, P90=0.02082435238344257, P91=0.02521412274597231, P92=0.0197347119954833, P93=0.005328394493403976, P94=0.0065292522112420474, P95=0.025644863263902327, P96=0.019435701094237225, P97=0.014861413892586836, P98=0.00855323686802945, P99=0.0033512495620343853, P100=0.0035481321027278057, P101=0.010164785710372357, P102=0.014556691700887062, P103=0.0094859836912471, P104=0.005654078955593065, P105=0.001963803418745957, P106=0.0024732511152996293, P107=0.0016109462940917263, P108=0.0022160001890763658, P109=0.0025568419854221326, P110=0.0013703023968429815, P111=0.00057108556932671, P112=0.0005461182333814737, P113=0.000517669398318415, P114=0.0006047080206453461, P115=0.0005327638251962301, P116=0.0003716414335265876, P117=0.00047383938703674205, P118=0.0011256897139539255, P119=0.0003581154733348876, P120=0.00029549156221493297, P121=0.0010983377159879699, P122=0.0009334712417801106, P123=0.0005028497206246543, P124=0.0009629664977920424, P125=0.0024107471749590073, P126=0.0027839422264628504, P127=0.0008316370033925985, P128=0.0012348014668514121, P129=0.00046636797450333114, P130=0.0012935716806110945, P131=0.0002017935204290267, P132=0.0006066407446102656, P133=0.0003870776811169948, P134=0.00018952955861302484, P135=0.0004196126039372625, P136=0.00010555539538480931, P137=0.00027613500968421306, P138=0.00014537331537072435, P139=0.00012285946878074, P140=3.32698319690608e-05, P141=4.1244931108157e-05, P142=6.177742062674565e-05, P143=0.00012872517553405054, P144=9.467017164798009e-05, P145=0.00014710904978071983, P146=0.0002594779388465186, P147=0.0004709421249175018, P148=0.0002354177895864655, P149=0.0004041963805623826, P150=0.0005547379123745325, P151=0.0003900195953986644, P152=0.00038158529912058576, P153=0.0004200004772711379, P154=0.00038086165292274254, P155=0.0003300628659171568, P156=0.00015678929315489834, P157=6.47871409422913e-05, P158=5.8757540968644575e-05, P159=0.0001469816350760847, P160=0.00019205748533603637, P161=0.00022806413466648854, P162=0.00026898252382581434, P163=0.0002869217066669586, P164=0.00021168403435217878, P165=6.49454712578043e-05, P166=6.672708391973114e-05, P167=0.0001435551358674908, P168=0.0004702663157080305, P169=0.000861094342968468, P170=0.0008104788786102822, P171=0.0005833349385105943, P172=0.00034194914713519875, P173=0.0002595759249041404, P174=0.00020067893785113226, P175=0.000205470044095864, P176=0.00020327596725014336, P177=0.00015600646440543743, P178=9.259707336128319e-05, P179=8.465897121203874e-06),Row( P0=-480.0758962498188, P1=43.09487455140312, P2=-36.869961242990236, P3=-6.408367516814514, P4=-22.569478729945953, P5=-20.58811656326469, P6=-10.961365773344099, P7=-4.9659326633311345, P8=-2.0273576768422688, P9=-11.526519856339055, P10=0.14881849948988396, P11=4.244128146868404, P12=-5.142386140900678, P13=2.016438306979174, P14=-1.4760668095914375, P15=-1.1655236459500065, P16=-0.8480155136960916, P17=3.6354354818120256, P18=11.198058066523167, P19=5.788329682471945, P20=5.963289984385333, P21=2.1692368271085667, P22=1.9248470673494174, P23=-0.4021873537822187, P24=-0.5337314983226517, P25=-0.06503639101708195, P26=-5.181114153044039, P27=0.32243386129822355, P28=-4.725225094687962, P29=-2.879752993835803, P30=-0.8250526240523697, P31=-0.30783525873602524, P32=-1.2791044441902502, P33=-0.0753237322518233, P34=-2.9123601573054128, P35=-0.4047724417009452, P36=0.3455966264888175, P37=0.6471523169241763, P38=-0.374669879324515, P39=-1.5386499400106313, P40=0.6112634332548439, P41=0.5452752553886856, P42=0.5087392792136909, P43=0.4929642511464889, P44=0.5111453598919711, P45=0.5431731148426723, P46=0.5848095060808113, P47=0.5728648383774888, P48=0.5903202447267643, P49=0.6231065054585607, P50=0.6455912155656276, P51=0.6261090584092466, P52=8.428957678330002e-06, P53=2.0633459866250965e-05, P54=5.8958183095783476e-05, P55=0.00018019498382628775, P56=0.00027827007291995003, P57=0.0002436155014798639, P58=9.588110469347085e-05, P59=8.033808913070915e-05, P60=0.00017773954446588975, P61=0.0014409681919914858, P62=0.015236217570067786, P63=0.03843337295199141, P64=0.09436034505909299, P65=0.41200263322804437, P66=0.6051743488686613, P67=0.3352847114580396, P68=0.12647116018586646, P69=0.23982883734780094, P70=0.22756009886570555, P71=0.13988245058820445, P72=0.2005272926594801, P73=0.25546990566745487, P74=0.5258262097286784, P75=0.5031342628567084, P76=0.6222781766766663, P77=2.786082912688084, P78=1.9591269240435907, P79=0.49419863212434584, P80=0.43380721634403135, P81=0.2647938058954815, P82=0.14796520940493263, P83=0.06257949590258553, P84=0.06953913296666127, P85=0.10255334000299672, P86=0.09878280894870727, P87=0.10163647736250124, P88=0.12090658668313271, P89=0.09272332446081405, P90=0.1271215829247276, P91=0.24371111457582179, P92=0.3446173326549838, P93=0.430591943774388, P94=0.9619471948862836, P95=0.8224165960877702, P96=0.3080359873954801, P97=0.5624997920196346, P98=1.4301212811469945, P99=0.5933284391414523, P100=0.04045618224148416, P101=0.033967777619620415, P102=0.06945574370645755, P103=0.05902710148866974, P104=0.037515834377257445, P105=0.07056934696497619, P106=0.20866478391140697, P107=0.23603762009986362, P108=0.15513184066180882, P109=0.13856372417088994, P110=0.19032995258277208, P111=0.2913357578270896, P112=0.11557991154514717, P113=0.022600057468970515, P114=0.05456240022265877, P115=0.07959222611996061, P116=0.0680284548634445, P117=0.041748357943832824, P118=0.019112265411368543, P119=0.03912128379579039, P120=0.07088902230521066, P121=0.046031986478898404, P122=0.028424565494894305, P123=0.039203004661246016, P124=0.0313162915792814, P125=0.024837642175612365, P126=0.018221487619543196, P127=0.016452846262600733, P128=0.012390697478029442, P129=0.013072704256416778, P130=0.012413077170477743, P131=0.01141557508969912, P132=0.006789916329266932, P133=0.006662199327996413, P134=0.004848357074790403, P135=0.00578675633256331, P136=0.004937292351244236, P137=0.00462710284699779, P138=0.002573591602377066, P139=0.0012650465234499803, P140=0.0011060936831836204, P141=0.001066272569222542, P142=0.0008026782459553296, P143=0.0011052503195714918, P144=0.00299590122016082, P145=0.0030296782100715128, P146=0.0020583563707742645, P147=0.0030989291694123204, P148=0.002949549966805406, P149=0.0014847835955746436, P150=0.0015315824962496543, P151=0.0019603943553175764, P152=0.0021951722464111827, P153=0.0029617028053672834, P154=0.0031940763182255654, P155=0.001462385166344561, P156=0.000769266244121793, P157=0.0005546531407149345, P158=0.00043729165047512854, P159=0.0006811545833349964, P160=0.0007382111591553467, P161=0.0021063592001014163, P162=0.0016348500002153887, P163=0.0017144587524835653, P164=0.001296193569111959, P165=0.001051214326438044, P166=0.0010343337261259039, P167=0.0008277844684649258, P168=0.0007750503422612536, P169=0.0010748531261832734, P170=0.000661339724615578, P171=0.000800097942824516, P172=0.00036540614148175554, P173=0.00014569914991180872, P174=0.00013232609143006474, P175=0.0002390868543503086, P176=0.0002496726777372466, P177=0.00023984918823423545, P178=0.00016797887730060312, P179=2.5442558961108337e-05),Row( P0=-478.0547151804938, P1=34.95988344033045, P2=-41.20382752573931, P3=-15.789468133001334, P4=-20.249413517450325, P5=-20.281605029036015, P6=-10.884851598788277, P7=-8.801352818198934, P8=-3.3531129972389913, P9=-12.62460334392392, P10=-5.628352043906779, P11=8.812893539798308, P12=-1.247675179597898, P13=9.958124605518725, P14=10.631631210538618, P15=6.10065958324905, P16=6.119985204878324, P17=-0.17425016760815432, P18=3.041418369435554, P19=0.18392272843749696, P20=2.0058265126116406, P21=2.7383001383750374, P22=0.48199753577320714, P23=-2.445608005552431, P24=-2.910929632565158, P25=-1.1186375998177658, P26=-3.21489580959223, P27=2.8861024523528003, P28=-2.706463785735373, P29=-2.4908540720232306, P30=-2.2613252425318517, P31=0.11961635405000785, P32=-1.6728220518527321, P33=-1.1517188855366096, P34=-1.9451838833738804, P35=0.09078261676561918, P36=-0.9150182776885735, P37=-3.1534894590231266, P38=-1.667977053523842, P39=1.0089553463972263, P40=0.5866593903069525, P41=0.4884722856959268, P42=0.4999181108902945, P43=0.5449888875807141, P44=0.5117783164776532, P45=0.48630553888730205, P46=0.4513032384876453, P47=0.5026282692392035, P48=0.50514160333063, P49=0.49794004390351465, P50=0.5594470019209987, P51=0.5940193938193609, P52=6.055665174286427e-06, P53=1.671615659132519e-05, P54=4.813965332497898e-05, P55=6.31409437664089e-05, P56=7.973968080219452e-05, P57=6.365606727151804e-05, P58=3.460829963109787e-05, P59=3.525025676784397e-05, P60=7.742518998972102e-05, P61=0.00013531176786955285, P62=0.00042545995030342687, P63=0.004781045902983176, P64=0.05642451046369586, P65=0.2772287272748213, P66=0.7546002134440487, P67=1.481793118495852, P68=0.6831038155855834, P69=0.5319697234639462, P70=0.8076564392329877, P71=0.6755703530694449, P72=1.5886689581657856, P73=1.4740971423944849, P74=0.55723082369554, P75=0.9003100655246244, P76=1.9126403701993804, P77=1.693328197694189, P78=0.16166736830242814, P79=0.044395944893452645, P80=0.10298705713854628, P81=0.09452955808673062, P82=0.07090201158528314, P83=0.155954274163473, P84=0.3214078691637724, P85=0.3152949200672955, P86=0.16921123650341727, P87=0.5263131079487928, P88=1.4283503829791195, P89=1.121795367896372, P90=0.7457064959638189, P91=0.816930602569009, P92=0.8873068637272283, P93=0.23872112436746468, P94=0.2280737074883184, P95=0.3651062610923047, P96=0.3138751025159683, P97=0.7517180210938134, P98=0.5794366142891677, P99=0.20535001825847612, P100=0.8400537908645385, P101=0.9452993161501132, P102=0.8042268471074318, P103=0.3404325077183306, P104=0.6545736962575915, P105=0.17102305727744613, P106=0.06408893517683023, P107=0.062437077089367125, P108=0.053978414857687555, P109=0.15699126185262988, P110=0.16916979291473253, P111=0.04837582413460753, P112=0.05893574327178267, P113=0.0788880211552538, P114=0.04321369056605277, P115=0.056608979695028476, P116=0.11155474510007979, P117=0.05485751653884869, P118=0.04063518153148802, P119=0.028960524911472198, P120=0.013043614176745096, P121=0.09870660511855604, P122=0.0666125805114979, P123=0.03688489604694453, P124=0.03288000587539523, P125=0.05666708828489549, P126=0.07368949699991781, P127=0.032593960461301995, P128=0.009184850853628908, P129=0.011657237304746049, P130=0.01693756559810232, P131=0.010790482714200937, P132=0.0019290352954099183, P133=0.002723136144016709, P134=0.004196039926052819, P135=0.0022434461177521796, P136=0.002702182229433059, P137=0.005983058738619949, P138=0.004211667174306626, P139=0.0032434605922293134, P140=0.0027943894995970144, P141=0.0016299540865241742, P142=0.001259959667163988, P143=0.0026044242229220894, P144=0.0026363000538862993, P145=0.00383320932461815, P146=0.0028228097004873143, P147=0.0033838510284551204, P148=0.002701698971046018, P149=0.0014426364403464988, P150=0.001053784107888259, P151=0.0012055651765311276, P152=0.0009918774795771178, P153=0.0007321619431032389, P154=0.00036648157541037786, P155=0.00022545570863429326, P156=0.00030772520218263176, P157=0.00043016752094321404, P158=0.0005471105553474352, P159=0.0008644358231927609, P160=0.0011689885454451954, P161=0.0010235630844748912, P162=0.0015090112107727176, P163=0.0019616634644572415, P164=0.0029119558884739215, P165=0.005075651566034665, P166=0.006321411649055573, P167=0.004297741767758549, P168=0.0017964847225630492, P169=0.0005921531331701815, P170=0.0007248972929569535, P171=0.0007776960082614718, P172=0.0005768010051844908, P173=0.000527860704922876, P174=0.0003476171623427457, P175=0.0003636213887029523, P176=0.0005830395543983784, P177=0.0008893117829228582, P178=0.0007540813584809369, P179=0.00011918904478501512),Row( P0=-501.5050146925076, P1=27.058688307238356, P2=-34.434271118584356, P3=-5.147292633688401, P4=-21.84375419655511, P5=-20.43819325894907, P6=-5.7189061315133305, P7=-6.7273214234901975, P8=0.2887473210123172, P9=-6.420870936062215, P10=1.55446570839104, P11=15.856557920611369, P12=8.350458705288336, P13=8.870131995732292, P14=9.773397466243043, P15=-4.290272859350675, P16=-1.8434829193659277, P17=-4.826182289941519, P18=-0.8733839695433219, P19=3.5515151933795153, P20=-4.470305283445766, P21=-3.158123655773803, P22=-1.5797192535600184, P23=0.5632763726576947, P24=2.6402267306253515, P25=-2.311817895528147, P26=-8.243075213395148, P27=-0.9842375381712933, P28=-4.117916806851257, P29=2.901024088923316, P30=-3.0712415652122007, P31=0.6218766284791196, P32=0.7288490815808301, P33=1.5433587697773596, P34=-0.3523973303606162, P35=-1.4613926674794733, P36=-1.0257212900779156, P37=-1.2049735681358638, P38=-2.463108894856023, P39=-1.4036633106704868, P40=0.6497563036307685, P41=0.5909377342939004, P42=0.5892193860463087, P43=0.5720891173247403, P44=0.6184811378571113, P45=0.6457961072807792, P46=0.7190459668834691, P47=0.6767448469315178, P48=0.5931930804750389, P49=0.5540025446625905, P50=0.5830492665519611, P51=0.6548204364115733, P52=5.030046723240516e-06, P53=2.030489577259981e-05, P54=0.00011575893838242408, P55=0.00023914227414221346, P56=0.0002923123097312811, P57=0.0002939910166810157, P58=0.0001583421577864447, P59=7.990704350934973e-05, P60=7.925137557875121e-05, P61=8.026671931847694e-05, P62=0.00014544861370118038, P63=0.00017894022526644736, P64=0.00018424993016349873, P65=0.002032609958489302, P66=0.02515509989899297, P67=0.11831346380396486, P68=0.18743115308976005, P69=0.20958397616516575, P70=0.2903683546259687, P71=0.4639240465348428, P72=1.4394529495112873, P73=1.833276940507772, P74=3.169509090290153, P75=2.7421278344942515, P76=2.105516362363698, P77=1.6269125450480064, P78=0.5832576611312117, P79=0.12641800418088672, P80=0.01788320266155682, P81=0.004742534316039239, P82=0.0191987542439854, P83=0.14761065488605302, P84=0.19795422876226781, P85=0.11623497053858356, P86=0.05077165321507342, P87=0.05597491621703289, P88=0.09175211678361099, P89=0.14249770122902586, P90=0.3971106636591343, P91=0.5890090666689622, P92=0.3496000629524979, P93=0.21718958500009716, P94=1.4927421127242373, P95=2.186593762216021, P96=0.8325872633523932, P97=0.23848930811221458, P98=0.09381216546869446, P99=0.01837070381622468, P100=0.012306054826726696, P101=0.018646250742392106, P102=0.057547850963003785, P103=0.06278371676776975, P104=0.05621335687621282, P105=0.09008578278387262, P106=0.18595759311806098, P107=0.6534536322446172, P108=0.3536498011654282, P109=0.12857990563231167, P110=0.028123736490411028, P111=0.01862135484507869, P112=0.03568599168966341, P113=0.030996027248883985, P114=0.02376027727978792, P115=0.02137677150605751, P116=0.012643297757727785, P117=0.04710929978183758, P118=0.05074200706601578, P119=0.02967111570185832, P120=0.009126690693473641, P121=0.013868925216295938, P122=0.029213749153777495, P123=0.026506636859230402, P124=0.03212105339436276, P125=0.037457723978682744, P126=0.038793657195188604, P127=0.025734367152471915, P128=0.015084808985602636, P129=0.017666115995836527, P130=0.009953754882975336, P131=0.00798522607657684, P132=0.010224525595049118, P133=0.01140365283252075, P134=0.0057802489935467345, P135=0.006550825984177699, P136=0.004405001884263643, P137=0.00558520868284494, P138=0.0025065250317852926, P139=0.002210392874357439, P140=0.002558772520588293, P141=0.0027506688434209003, P142=0.0025116199114413196, P143=0.0061156417176811435, P144=0.0062943900222832165, P145=0.006614204448926808, P146=0.007526949348843834, P147=0.007467137796549152, P148=0.0035075346214044636, P149=0.0024880360465277507, P150=0.0015564018214998111, P151=0.0007146657363760312, P152=0.0005711974196023201, P153=0.00040018694406731523, P154=0.00056523053354453, P155=0.0006692484371206524, P156=0.0008675086792519543, P157=0.0010841237318998818, P158=0.0014807079475751233, P159=0.0017627391568889005, P160=0.0015364571731421475, P161=0.0005934627043875524, P162=0.00029317780323735466, P163=0.00023038596019242513, P164=0.00045995467641888715, P165=0.0009418307036431548, P166=0.0008247975822923425, P167=0.0005997935923940381, P168=0.0004711245748818286, P169=0.0002893218434363335, P170=0.0003056104672132704, P171=0.0002755688868931157, P172=0.0002300947588308603, P173=0.0003829477563059197, P174=0.0004524915891674729, P175=0.0006369833573367115, P176=0.00047362180156679983, P177=0.00035151721585938357, P178=0.00020411155198566712, P179=3.0370844106210858e-05),Row( P0=-535.8386037439578, P1=30.809768009607502, P2=-31.834332847455844, P3=-3.112970610138139, P4=-22.032448156344397, P5=-20.814771092635524, P6=-8.455573989164698, P7=-1.440665799523406, P8=0.7206507130869825, P9=-5.824222149180814, P10=7.734952872351641, P11=14.475228196953305, P12=4.774950197359313, P13=10.631863182417632, P14=6.724109476983367, P15=-1.6560929979261432, P16=2.145425669388139, P17=-6.449445701951447, P18=2.26328320216332, P19=4.204133763813503, P20=-2.4753227353377714, P21=4.897766729260688, P22=0.7916256294732759, P23=-5.068295269415862, P24=-4.471668990400001, P25=-1.5688738618946827, P26=-0.044648522434234485, P27=0.40304292844842204, P28=-0.22542620623316567, P29=0.4065919969638374, P30=-0.3529521292934657, P31=0.35371517448775386, P32=-2.7847354827964246, P33=0.6215864260902757, P34=-0.5352717865372395, P35=-2.5128702787601602, P36=-4.1889049708589, P37=-0.22285463988702559, P38=1.5187526886156506, P39=1.3355991483878782, P40=0.5699838636787043, P41=0.5387255087116832, P42=0.534654084564961, P43=0.5666144950765521, P44=0.6026973882959755, P45=0.5559020686202427, P46=0.5212529866661535, P47=0.5279676403356043, P48=0.6474439561480094, P49=0.6733348254256937, P50=0.6038296160414713, P51=0.5825887835470837, P52=5.573003388258167e-06, P53=1.8444429050442217e-05, P54=3.827923221000056e-05, P55=3.072767990147028e-05, P56=5.116130040756506e-05, P57=5.817846889608069e-05, P58=2.7030567462741297e-05, P59=1.3939389456404788e-05, P60=2.4770619688297596e-05, P61=5.56474241570782e-05, P62=5.6033322536827227e-05, P63=4.402377125012781e-05, P64=7.158899568765667e-05, P65=0.002297728034669228, P66=0.04403509482782055, P67=0.1416793009589645, P68=0.15151164608195683, P69=0.27897902587270285, P70=0.3272396370603704, P71=0.9085078426323124, P72=2.0604859155231905, P73=1.2992454578856085, P74=0.5475196988683321, P75=0.5330257954389048, P76=0.6054013251224949, P77=1.9369826168608943, P78=1.7487806284173155, P79=0.5084554009604112, P80=0.046945881612785144, P81=0.006738224784065966, P82=0.004500113038208818, P83=0.004584055033605468, P84=0.006517762128608653, P85=0.0071731933087460014, P86=0.00770787229715637, P87=0.031345421228223216, P88=0.07421004684482205, P89=0.25084824350235296, P90=0.8087982875907209, P91=0.6263197304807998, P92=0.33650247964043795, P93=0.1238638664141059, P94=0.09826340289687692, P95=0.08481294669109525, P96=0.040265328752601394, P97=0.07841643063636418, P98=0.20696975839076728, P99=0.17483619640653814, P100=0.08836188844427666, P101=0.05407184536078772, P102=0.10713260149193278, P103=0.06766949492855304, P104=0.015419487362866136, P105=0.01071104442898255, P106=0.010080126528605722, P107=0.009162900672779539, P108=0.015529078484406747, P109=0.007833944577330597, P110=0.0049396598889826565, P111=0.020156540246285697, P112=0.025438694539515527, P113=0.007122058974227403, P114=0.007899338156198144, P115=0.01101872238717849, P116=0.0034915505041733267, P117=0.0018333404695690767, P118=0.020740098117863818, P119=0.017239209116378534, P120=0.025728296204926584, P121=0.024023768797504726, P122=0.0057408175105285696, P123=0.037098079227931575, P124=0.07865874071700672, P125=0.012864335876374608, P126=0.01717797429732114, P127=0.010704913470930647, P128=0.011344590474837612, P129=0.008492462377827827, P130=0.008128103048074781, P131=0.0055709267692328574, P132=0.006939590657441974, P133=0.006886574834809751, P134=0.0033557495261224775, P135=0.007255784796911557, P136=0.007048791363250817, P137=0.0013917934129014102, P138=0.0006248956700276395, P139=0.00035608892695144075, P140=0.00011102006071727987, P141=0.00012505418248462695, P142=0.00039466734357214916, P143=0.00030877484655432625, P144=0.0006190078059536843, P145=0.0015912025715086257, P146=0.0013807807463928866, P147=0.007271953706595907, P148=0.005663240407200111, P149=0.0033608808021598742, P150=0.0019872852850133928, P151=0.0008851186741523479, P152=0.0008291084523531727, P153=0.0005758536638586863, P154=0.00022593113498597323, P155=0.0001463222440460198, P156=0.00012813538084475182, P157=6.388255450268728e-05, P158=0.0001286651390276789, P159=0.00028653307624534836, P160=0.00034671602454006085, P161=0.00043770159875071413, P162=0.00045856786781063884, P163=0.00024821138725104224, P164=0.00011736585812245248, P165=6.37899772148922e-05, P166=0.00011551066711472598, P167=0.0002962932753190275, P168=0.0003844001576614231, P169=0.0003927686561307673, P170=0.00029290888094386737, P171=0.00019471197051367875, P172=0.00013755113202686717, P173=0.00015553973838391106, P174=0.0001257401228265047, P175=9.249812660371842e-05, P176=8.049423113596626e-05, P177=0.00010161135857750031, P178=6.239170547518517e-05, P179=7.089790674361597e-06),Row( P0=-624.5704003840762, P1=60.29127307881688, P2=-5.575124278514004, P3=0.5378144120880427, P4=-8.545783975596668, P5=-10.565452873112449, P6=-6.559003677138708, P7=-2.7677870021439888, P8=-1.2515147380958824, P9=-7.481379550747051, P10=-3.966823401032496, P11=-4.106320615298245, P12=-7.926410007816106, P13=-7.996073990492033, P14=-2.9395551707182723, P15=1.1421724497305283, P16=1.5477077409124018, P17=11.194010036962665, P18=13.09837954049792, P19=14.929809710939743, P20=11.97450592060473, P21=6.9544885226270505, P22=4.036745787253993, P23=-3.3315769557084653, P24=-1.1339452786310957, P25=-1.3554630004062214, P26=-4.121798186863991, P27=0.40596878843322326, P28=-6.451097940650751, P29=-4.5419749013017485, P30=-4.574454342411897, P31=-1.6471907559801384, P32=-4.636419906454191, P33=-2.725431906932665, P34=-5.945646790409112, P35=-2.888513757201681, P36=0.20505908155147368, P37=1.8849685802721032, P38=-2.2168159220193338, P39=-2.2069384645123207, P40=0.5147745367677911, P41=0.4802063476639296, P42=0.48630095405793394, P43=0.4788901779618204, P44=0.48786426528050747, P45=0.5378108162180054, P46=0.5846466676962859, P47=0.6136191848331323, P48=0.6251208767854123, P49=0.7209628377900066, P50=0.763439200815959, P51=0.6440639896197682, P52=4.039667145858653e-06, P53=1.2016219660475557e-05, P54=2.822721729738418e-05, P55=2.4355281831604813e-05, P56=1.820579590245782e-05, P57=1.327973036142538e-05, P58=5.021306716229548e-06, P59=4.947773882444821e-06, P60=2.6313996833494327e-05, P61=0.00015384417564318922, P62=0.008746785147881079, P63=0.03780959869566401, P64=0.08111838295967708, P65=0.3335618196602348, P66=0.44485502485737066, P67=0.16674162811064772, P68=0.01779716793300148, P69=0.0029520324142898366, P70=0.0016118082812894947, P71=0.0005887349177292193, P72=0.0002733885911686057, P73=0.00030670335304500894, P74=0.0038340073744656, P75=0.010405498807153522, P76=0.012647858667999973, P77=0.03617729150613203, P78=0.07244824894799382, P79=0.10672854059487563, P80=0.15120010079808774, P81=0.17324406480889748, P82=0.10776652253899041, P83=0.022677752302850542, P84=0.011192805058177227, P85=0.005699220971476302, P86=0.0029925102564986694, P87=0.00905534666548418, P88=0.015820194244860836, P89=0.011061017815250004, P90=0.008442761570806543, P91=0.014856087352453284, P92=0.03956843078746973, P93=0.01244381872896272, P94=0.004486504943104034, P95=0.0028100142979962653, P96=0.0009563423328366445, P97=0.0006834375899841632, P98=0.004474030250688408, P99=0.008563414336632507, P100=0.012123892356958078, P101=0.018621133816096844, P102=0.004325202247518655, P103=0.0006167279079766869, P104=0.0003254829195951928, P105=0.0022391457523371103, P106=0.0024491013019247465, P107=0.001718609546631208, P108=0.003716217524385647, P109=0.001012312084356154, P110=0.00027999126474853456, P111=0.0003052417052902909, P112=0.0014226486448139833, P113=0.0014839901341471631, P114=0.0007541557404821588, P115=0.00033702145325431197, P116=0.0001852501618767398, P117=0.00015714981882320342, P118=0.0015343962746419118, P119=0.0007585950421258882, P120=0.0003132689155517927, P121=0.0002925426746863375, P122=0.002123774984527665, P123=0.001074560990950803, P124=0.00039620744011114814, P125=0.00047944312411994216, P126=0.0009130266478830843, P127=0.000537532488936151, P128=0.0004556362082738578, P129=0.00015520367683115278, P130=0.00013713265843838619, P131=0.00011924500763428799, P132=9.171033161489767e-05, P133=3.578962612492526e-05, P134=0.0001103615794580783, P135=0.00014259658035747143, P136=9.225341367551196e-05, P137=0.0001187528243287941, P138=4.780184720411575e-05, P139=5.3282446016517314e-05, P140=3.1096620688025284e-05, P141=3.4218850940429025e-05, P142=4.978313298416471e-05, P143=5.7076725668393065e-05, P144=7.585122476951457e-05, P145=0.00013389605334378086, P146=9.559659852357403e-05, P147=9.801850850422582e-05, P148=0.00013786204780727446, P149=9.442591788769951e-05, P150=3.333207209740911e-05, P151=8.173045652148273e-05, P152=0.00013354050164953837, P153=0.0002644179536695401, P154=0.00029533824123958053, P155=0.00033929207049996485, P156=0.0004230608950114334, P157=0.0004216927241286213, P158=0.0003495060400930126, P159=0.00019836308090651574, P160=0.00013898901331125838, P161=6.131566487507857e-05, P162=9.883553120576626e-05, P163=0.00014217864684128344, P164=0.0001749732630597457, P165=0.00014406201452141207, P166=9.867380309890073e-05, P167=4.7719941071064176e-05, P168=3.566518927531259e-05, P169=0.00011551506668708727, P170=0.00025500483478736545, P171=0.00022848902081500227, P172=0.00013332371922440366, P173=9.406795166815337e-05, P174=0.00012512164996624184, P175=0.00025518347648880926, P176=0.00023264567001437338, P177=0.00020172517342454706, P178=7.78575853501136e-05, P179=6.430876470190553e-06),Row( P0=-628.0891994968109, P1=54.06531392399979, P2=-7.951802350559492, P3=-0.4085965085193153, P4=-7.706360462052973, P5=-5.150735706359884, P6=-2.229541429553277, P7=6.282644701088706, P8=4.849957725530818, P9=-9.094841685432547, P10=-3.9255687816234857, P11=-2.5957314300074605, P12=-10.466530803725673, P13=-3.1387508697324216, P14=0.7896199035297289, P15=5.152728123357295, P16=5.788437618032912, P17=4.821198550128407, P18=8.65186426278125, P19=11.703281009973248, P20=6.48881795307881, P21=1.8727279225951798, P22=-0.5676311350435475, P23=-5.720801379818962, P24=-4.057383449699749, P25=-2.9441284709074216, P26=-3.807514543452457, P27=-0.9685077137185467, P28=-4.42639179480856, P29=-4.602348363738757, P30=-3.7593823993846565, P31=0.37839891829628686, P32=-5.181705244452997, P33=0.024023152043809195, P34=-3.536906459290945, P35=-0.3621246941994174, P36=-1.0758766653928438, P37=-1.0388752818431204, P38=0.09989257872590789, P39=-0.531661399750107, P40=0.6597046902308119, P41=0.608317921085935, P42=0.5823406264037917, P43=0.5901182457293933, P44=0.5600615276772627, P45=0.5376564550944998, P46=0.520417383002733, P47=0.5921927202438954, P48=0.661783999875014, P49=0.6983858669180171, P50=0.7149637415895399, P51=0.7159229268173101, P52=5.421162551023043e-06, P53=1.7023191349412162e-05, P54=6.647110235619083e-05, P55=0.00012785537994503573, P56=0.00016928066059627606, P57=0.00022340569011792768, P58=0.00013351480413462488, P59=8.68646519634009e-05, P60=7.891979730214863e-05, P61=0.0001503305845724467, P62=0.0015504165244758239, P63=0.01001912224720715, P64=0.054269249514624016, P65=0.21517933793255484, P66=0.29461106728853875, P67=0.1951438657006909, P68=0.08180819866820502, P69=0.061182268118133334, P70=0.0518702091167803, P71=0.023382536481726314, P72=0.004680618863569238, P73=0.0014494864274948698, P74=0.0010191464983308062, P75=0.0018355261293394563, P76=0.005694875302858953, P77=0.036163613278608106, P78=0.07616575149154539, P79=0.06803192653031588, P80=0.07762780268733553, P81=0.14063546045206976, P82=0.08672913869540674, P83=0.015905778373300657, P84=0.018658030637624258, P85=0.024299952726740105, P86=0.009151845396225518, P87=0.004308245693687912, P88=0.0043924692843718, P89=0.0041587150136389, P90=0.008294446074208728, P91=0.019279957836154927, P92=0.016384398464346236, P93=0.006216867663833599, P94=0.007797639324890403, P95=0.02168619901256511, P96=0.010449912864410263, P97=0.0007804991063866997, P98=0.0007600937423566219, P99=0.004084813796131522, P100=0.0038377815159792864, P101=0.003625445664612855, P102=0.004130164561921865, P103=0.00498832162818461, P104=0.0161272019529764, P105=0.004993667695776144, P106=0.001489586026928202, P107=0.0007351626521790303, P108=0.0013333521892355616, P109=0.00149373858685903, P110=0.0005041510989549193, P111=0.0005137485049762992, P112=0.0007768539124877534, P113=0.0004853965672679486, P114=0.0003003703988024429, P115=0.0008481499984435423, P116=0.0004695862566914934, P117=0.0005042828068792861, P118=0.0004536261951511402, P119=0.00042036506687729067, P120=0.0006427581836288724, P121=0.0010256571363049887, P122=0.0007926312278720087, P123=0.0007586867443369398, P124=0.0009603916541077994, P125=0.0007893935843884578, P126=0.0006450176804550433, P127=0.0004965462652778546, P128=0.00036359924989747846, P129=0.0002408996963977667, P130=0.00015937577067872145, P131=0.00017896266907962142, P132=0.00021236128989571035, P133=0.00025188731305502066, P134=0.00028402005497848575, P135=0.0002801877959259992, P136=0.00016301466551615313, P137=0.0001293037004545871, P138=7.173711785289306e-05, P139=3.738142050976229e-05, P140=3.876407341138067e-05, P141=6.456007393975775e-05, P142=0.00010680768145222491, P143=0.00026649238428444813, P144=0.0002869980611082261, P145=0.0005092245770654414, P146=0.0006969791843535368, P147=0.000982176152984999, P148=0.0004892787745598251, P149=0.0002241829194859238, P150=0.0004754373141834675, P151=0.0008580557758420788, P152=0.000623970756450254, P153=0.0003625295527650229, P154=0.0001660414980953707, P155=4.81625484806981e-05, P156=3.438548956300003e-05, P157=8.427939585080238e-05, P158=0.0001640952307994237, P159=0.00017144352075627555, P160=0.0002762077223318926, P161=0.00021020899090045593, P162=0.00019465529944956786, P163=9.310397727381522e-05, P164=4.580035964005743e-05, P165=5.217244895625314e-05, P166=0.0002139492916600608, P167=0.00026916693811563475, P168=0.00034254844700816716, P169=0.0002928441839695471, P170=0.0001435167916405929, P171=4.3998327034124554e-05, P172=6.110077376487428e-05, P173=0.00013944656174400683, P174=7.460093963584514e-05, P175=6.568474981781721e-05, P176=5.150709678590449e-05, P177=4.8074066788245704e-05, P178=2.705968776608294e-05, P179=5.301241887769522e-06),Row( P0=-579.0081400464773, P1=50.91972987534937, P2=-21.074031694264495, P3=-4.681795182570166, P4=-15.805233409247927, P5=-19.092932148122788, P6=-2.230311496049708, P7=6.505157471174066, P8=5.640245363674899, P9=-9.20168845493087, P10=-4.751513148442672, P11=-3.541787011539266, P12=-8.11281322988469, P13=-4.591505189094073, P14=5.500707512721001, P15=9.247112893788815, P16=12.940043245909127, P17=13.804415211397238, P18=8.781355300059396, P19=10.903106324732093, P20=-1.6282161382441371, P21=-2.6727671441935006, P22=-2.0506112002379564, P23=-5.4055569702303, P24=-1.283995404504237, P25=-4.7593907950540855, P26=-7.844056687461802, P27=-4.211026661860145, P28=-6.5083723158931965, P29=-0.425086742346861, P30=-1.4450716707457956, P31=1.017131878717294, P32=-1.3784328742296352, P33=1.5204282801154472, P34=-2.9800765406282133, P35=-0.15650189264567518, P36=-1.7667827738126634, P37=1.7270400628266556, P38=-0.043186557959937075, P39=-1.5326981779717, P40=0.6798001329438864, P41=0.5547599597198261, P42=0.47233103142765787, P43=0.4201736146979589, P44=0.44774636531288425, P45=0.4275325242407128, P46=0.44206356896789506, P47=0.5233438825350855, P48=0.5925298060363337, P49=0.6444833471696905, P50=0.7249698640930442, P51=0.7327710077280567, P52=5.429948028565046e-06, P53=1.670704972606433e-05, P54=3.6639114788147763e-05, P55=4.95512231389345e-05, P56=6.434159529684414e-05, P57=5.914899084507109e-05, P58=2.576748531597765e-05, P59=1.4653258616339686e-05, P60=2.4651163276078078e-05, P61=3.3470031911643985e-05, P62=9.70794826131729e-05, P63=0.0009042145623957145, P64=0.011987601227768784, P65=0.17654399477957167, P66=0.4353654162139651, P67=0.3640153449551305, P68=0.3939833078083119, P69=0.36290266128067694, P70=0.07216308091279092, P71=0.004105523511204587, P72=0.0007661939053210951, P73=0.0005109266387938787, P74=0.00041197584510728805, P75=0.000580012310716631, P76=0.0005276790980393397, P77=0.0013362289447251725, P78=0.00563744483146572, P79=0.028391763990951866, P80=0.09287226582704199, P81=0.2153590435566281, P82=0.3516397977047019, P83=0.17672826489441254, P84=0.18771090087842868, P85=0.16393105196851043, P86=0.06836215124279316, P87=0.02937275973540972, P88=0.004300677619175742, P89=0.0012405237474120766, P90=0.0010240907100472463, P91=0.008457637033074521, P92=0.038302229103462974, P93=0.03810221287690923, P94=0.05335559059587806, P95=0.019618638812741417, P96=0.011954739537730368, P97=0.06584664568560164, P98=0.022666035820857886, P99=0.0016400823319402237, P100=0.0015829492272593593, P101=0.00415883401016251, P102=0.0037447134833076125, P103=0.005738978761327711, P104=0.0017534704967112204, P105=0.031041310122082475, P106=0.046999078822893066, P107=0.0028584570231916877, P108=0.001433458862419659, P109=0.0019342293761800422, P110=0.004742532910183258, P111=0.0008756627643628142, P112=0.011309028678439588, P113=0.015983252069433164, P114=0.0037022539614577498, P115=0.0045953242057757415, P116=0.0008134309726362611, P117=0.0005033738624509566, P118=0.006341596950839998, P119=0.005340019997182801, P120=0.0006332454872895383, P121=0.0006143144553397133, P122=0.0008992711180014445, P123=0.0027172712632685873, P124=0.000982773579240098, P125=0.00046517597768770447, P126=0.0003580228420552661, P127=0.0007924672926600803, P128=0.00068967023110149, P129=0.002052096539240022, P130=0.0008181689773897166, P131=0.0003765208318813488, P132=0.00023410486968549805, P133=0.00022413028646447197, P134=0.0006580521816835584, P135=0.00031027740495529816, P136=0.00029746648863352874, P137=0.0004148056520446936, P138=0.0002370036929305696, P139=0.00020882547790371396, P140=0.00016360890506198647, P141=8.500638251763534e-05, P142=9.585704194601304e-05, P143=0.00012904474971233706, P144=0.00020150885813401159, P145=0.00044089138467208, P146=0.0007827626222072209, P147=0.0005393511409884062, P148=0.0005837399064088638, P149=0.00021782686101117673, P150=6.781864590585345e-05, P151=8.443806688981145e-05, P152=0.00024533020467916835, P153=0.0005593582882875902, P154=0.00046150967003400444, P155=0.00044622513956199593, P156=0.0005674999421358684, P157=0.00029443633878557416, P158=0.00039501208874308046, P159=0.0006663778564622811, P160=0.00035697164055663403, P161=0.00027116413293535896, P162=0.00015780026321874106, P163=0.00014475642267642032, P164=0.00011513430205047078, P165=0.00012319253634839248, P166=0.0002855675093120754, P167=0.00023468041396628125, P168=0.00011468894227852822, P169=6.382070301562017e-05, P170=5.73911288790019e-05, P171=9.066281105347218e-05, P172=9.756781051440094e-05, P173=6.541420596683202e-05, P174=6.390809702487593e-05, P175=7.543700382568305e-05, P176=8.147367989789671e-05, P177=8.826671449074134e-05, P178=5.680516371592695e-05, P179=6.964967529980751e-06),Row( P0=-400.64466188592974, P1=20.065036554880315, P2=-44.68398187283618, P3=-11.204159358983143, P4=-28.197054639264792, P5=-28.366937373683836, P6=-7.33107956741009, P7=-9.347969453876576, P8=5.963151051047519, P9=3.3109281489910485, P10=4.790879442931123, P11=17.252437231396772, P12=-4.802039757858095, P13=-4.513862223128051, P14=-4.608767075083552, P15=-7.954567156355039, P16=3.610364153803351, P17=0.7600983467480015, P18=1.773568366451192, P19=3.6232478668029824, P20=-2.4373938217401414, P21=4.249214543768991, P22=1.1580283275693721, P23=-3.409524929755254, P24=-3.840015107689322, P25=-0.5895253789630674, P26=-4.171969605939723, P27=1.81890136134454, P28=-0.5160335521128444, P29=0.8837002268875684, P30=-1.001954695671402, P31=0.8395578236368044, P32=-0.36616600236264485, P33=-1.2905318970461905, P34=-0.9522124956048739, P35=-2.0348842709970567, P36=-0.8612554084812142, P37=-2.041334837592612, P38=-1.5130274114628566, P39=2.194026277115137, P40=0.5706413450075081, P41=0.5086540485320282, P42=0.4760242732028121, P43=0.4509727270846495, P44=0.45870409893208985, P45=0.4683320180798371, P46=0.5219007235632561, P47=0.5585266320127059, P48=0.6726868741588204, P49=0.728536716710183, P50=0.6882267136326224, P51=0.6422933629149137, P52=8.162043757051011e-06, P53=2.6354196956276065e-05, P54=0.00011443980860524154, P55=0.0001682553683092993, P56=0.00017680178313700013, P57=0.0001886459912816521, P58=0.00016118716201890667, P59=0.00024220947733429642, P60=0.0003068876990406869, P61=0.0004666474005684499, P62=0.0008397125662614241, P63=0.0017931170613093355, P64=0.003577882043681533, P65=0.009284033962488619, P66=0.016587457045752336, P67=0.022335236659949992, P68=0.10071849373354012, P69=0.4136289130775817, P70=0.3662635040160655, P71=0.17563122974408985, P72=0.29646030877744056, P73=0.42644338170268303, P74=1.6287549115154505, P75=2.9013753967978855, P76=8.284310317735992, P77=28.792470757370765, P78=31.668371719695763, P79=21.190374230777834, P80=19.198571232005737, P81=15.147547276265266, P82=15.371068746029687, P83=3.7845960178201152, P84=0.6003148309039125, P85=0.2223538401356914, P86=0.08789289175419993, P87=0.11128377064364815, P88=0.15251265810020065, P89=0.15908566664113655, P90=0.22904252781093803, P91=0.4677960729797108, P92=0.7622890359504266, P93=0.5510596133436204, P94=1.1771639649545027, P95=2.0735673876600114, P96=1.9110870826245208, P97=2.999183914950148, P98=3.3708216128432302, P99=1.5289608780104331, P100=1.4214257619349522, P101=0.933266839588725, P102=0.37564947579198676, P103=0.20984525552427077, P104=0.09071418283073317, P105=0.13793921638815765, P106=0.19747863362981116, P107=0.12497904645643859, P108=0.14053723790567757, P109=0.534675822196541, P110=1.1722874915414667, P111=0.8563400834447072, P112=1.2826704491732794, P113=0.3183523611721426, P114=0.15122590293844404, P115=0.1951982753549566, P116=0.26460365839802374, P117=0.43524295681524294, P118=0.37117957097609866, P119=0.16247698041234956, P120=0.1430734825442868, P121=0.12259618637733911, P122=0.17096943163660944, P123=0.7068493020605349, P124=2.4940321537980306, P125=1.667249916884077, P126=0.14708419401798106, P127=0.07399617108309342, P128=0.09092552048137245, P129=0.08830012544424265, P130=0.11015233335697516, P131=0.19049601423259122, P132=0.09085466004543914, P133=0.04441499174196047, P134=0.040687990578728524, P135=0.058031980426517034, P136=0.09368317870399774, P137=0.10118695405969992, P138=0.018935669973034155, P139=0.009568485294287978, P140=0.01343181479670629, P141=0.033828113274013236, P142=0.04311553550831579, P143=0.04568095662786364, P144=0.18075817770319855, P145=0.19909037910489769, P146=0.13671264678948053, P147=0.028959250471447432, P148=0.020291976063379447, P149=0.0160260821469353, P150=0.01035721026281839, P151=0.003163706044269058, P152=0.0032634636849382913, P153=0.0024602198832984635, P154=0.002406984446796992, P155=0.0015992226276358066, P156=0.001658290256389034, P157=0.002744541304291697, P158=0.0018095508067331554, P159=0.0008128913998553058, P160=0.0006246844492609889, P161=0.00047773969053356064, P162=0.0005898807345560219, P163=0.0015213824523757093, P164=0.0026376870308013837, P165=0.003357138862957702, P166=0.002274610604380954, P167=0.0012207238587847453, P168=0.0006892780982832785, P169=0.0007564816738336906, P170=0.0014781560959616653, P171=0.001511257588321587, P172=0.0014421266620563398, P173=0.0010954021926599842, P174=0.0008047597917034068, P175=0.0009407291986047765, P176=0.001407167327461635, P177=0.0015801319550287954, P178=0.00070548332607155, P179=8.191904012581225e-05),Row( P0=-608.6087853031092, P1=50.73042563351855, P2=-4.591555662569602, P3=-1.9515182959789752, P4=-9.33137542795058, P5=-9.201635981261385, P6=-4.188136236063994, P7=1.3806797546098444, P8=2.317408707372347, P9=-9.146636310989614, P10=-2.1238292722086753, P11=-1.5759965433823344, P12=-10.745993959271642, P13=-4.922716320713876, P14=1.7334862532200965, P15=3.7453523634007895, P16=9.370161772794168, P17=9.83584814560741, P18=11.029926807841195, P19=10.1672226105799, P20=3.181134554909021, P21=2.5974868216775797, P22=0.49293595605807267, P23=-4.567014044461046, P24=1.179110708304121, P25=-2.012567607900184, P26=-5.365026355329164, P27=-3.415672552606117, P28=-5.586125960329341, P29=-1.1419723084725844, P30=-2.344584099722693, P31=1.1754742055912597, P32=-4.462809313190804, P33=0.5812129654054724, P34=-0.6462114003264561, P35=-0.6662428484324668, P36=-1.9288475886870475, P37=-2.2533215091226526, P38=0.8003091364860153, P39=0.16465471588621458, P40=0.7292902276296698, P41=0.587589108490409, P42=0.5232306028556266, P43=0.5147790855959063, P44=0.5203239703377116, P45=0.5271155335664639, P46=0.5972125198792548, P47=0.6083977521121103, P48=0.6492765381290152, P49=0.6784136056277413, P50=0.6964163337674686, P51=0.7573454004639765, P52=9.307543318078434e-06, P53=2.304010528355952e-05, P54=5.1167831023778823e-05, P55=6.714211080889468e-05, P56=7.031782958055778e-05, P57=5.184305728892391e-05, P58=2.2841457330709168e-05, P59=1.269346975201251e-05, P60=3.0103699935877262e-05, P61=0.00022026708531427245, P62=0.0038508465765516916, P63=0.012974012283794415, P64=0.04881563926539732, P65=0.24160158088960035, P66=0.6318155275016316, P67=0.8021785955871579, P68=0.24545880321536298, P69=0.021955616497683295, P70=0.006934211351330213, P71=0.0019452926576264697, P72=0.0007167849376296732, P73=0.0005933476003951846, P74=0.0007211282926666293, P75=0.0014056829405725369, P76=0.002688944298421821, P77=0.029812509115576903, P78=0.12797749772742964, P79=0.1388483268704746, P80=0.09016875204083002, P81=0.07203886086729042, P82=0.09622769088828129, P83=0.16509121823906261, P84=0.179053163696048, P85=0.09262216056617163, P86=0.015491202467778584, P87=0.004570226795020316, P88=0.0022783808388581344, P89=0.0027121992827918344, P90=0.008256996483223292, P91=0.011364093104206135, P92=0.003535368223902723, P93=0.003463538917610026, P94=0.05727379375991675, P95=0.07520754263707254, P96=0.02080616050157594, P97=0.0013773969598209973, P98=0.0010963365622843693, P99=0.0023074688857321795, P100=0.00191651578012879, P101=0.0015005392373195875, P102=0.0035267098289263267, P103=0.0319958930648138, P104=0.018595068251018382, P105=0.0036521740918512287, P106=0.0020499593696179296, P107=0.0004284738297518239, P108=0.00016916548907395263, P109=0.0003901676386997328, P110=0.001994235189133048, P111=0.004903182473352257, P112=0.005693974306030863, P113=0.001082971665880299, P114=0.0001702651561361069, P115=0.000917899151651409, P116=0.0010983766470151388, P117=0.0007401570069494388, P118=0.0006307225681309106, P119=0.0002498221188510249, P120=0.00038972829244781764, P121=0.0008966174693164407, P122=0.0008492645359874688, P123=0.0006489259560757363, P124=0.000966614543513181, P125=0.0013520501268276908, P126=0.0009325167609748722, P127=0.00034075929648144177, P128=0.00018570659436167026, P129=0.00027029250338102056, P130=0.00018465530600177002, P131=8.8166882392631e-05, P132=0.0002351991734269319, P133=0.00030971650331804856, P134=0.00010188355598064817, P135=0.00012735365845607344, P136=0.00013158661655051206, P137=0.0001295434775031824, P138=6.795531153408395e-05, P139=8.213479463822661e-05, P140=3.611019825263266e-05, P141=5.43768303216991e-05, P142=8.343030807107743e-05, P143=0.00034695932843469365, P144=0.0006121200097605873, P145=0.001618329475456743, P146=0.0014864737894647127, P147=0.000869457040937664, P148=0.0010401579517807817, P149=0.0013580303674043757, P150=0.0009369252961811498, P151=0.00040150579128970214, P152=0.0003625497238906692, P153=0.00020437916237624573, P154=0.0001552855950641513, P155=0.00020593530784732117, P156=0.00022095908723431784, P157=0.0005725976791417395, P158=0.0007720007341662004, P159=0.0006937026825917062, P160=0.0007349235374077407, P161=0.0006612639683110207, P162=0.00041801827659581915, P163=0.00013872130370696413, P164=0.00010292309932264649, P165=0.00014647899414247382, P166=0.00030063377568046085, P167=0.0006007335802458392, P168=0.0008866048297317112, P169=0.0006669573743571907, P170=0.00014837267136002605, P171=0.0001555627900047761, P172=0.0002858518226325602, P173=0.0002530343994369866, P174=0.0002098039579560777, P175=0.00014528078299856547, P176=0.00015751531895885488, P177=0.0001925371246921593, P178=0.00011393385786977342, P179=1.1761135584580596e-05),Row( P0=-621.6957085871829, P1=63.86206143316044, P2=-16.421228635697172, P3=2.402826458322297, P4=-13.266510886513226, P5=-9.261076386259592, P6=-4.103933161320377, P7=-0.9186919960117711, P8=4.960389799545927, P9=-9.036264479644286, P10=-2.8333638826055774, P11=-1.1593145121064157, P12=-8.76505396508351, P13=-3.1449542237094987, P14=-2.926277917961717, P15=-5.043146710154884, P16=2.9100437536287798, P17=2.2026178251724837, P18=6.8512318304956095, P19=12.137628120953682, P20=12.060647725573178, P21=10.232186505790128, P22=8.587136605571926, P23=1.7748937269874037, P24=-1.298100042289517, P25=-2.5547082108557655, P26=-6.571550037984037, P27=-1.1349995015734182, P28=-0.2597906920525549, P29=-0.3438963502299378, P30=-2.7872713633363295, P31=-4.934988640476619, P32=-6.2714176000782444, P33=-0.40221458952449585, P34=-2.5896752344448757, P35=-0.6108347326249542, P36=-5.059972969112699, P37=-5.199626998846119, P38=-3.176830414399549, P39=1.34482206765665, P40=0.6155927383424317, P41=0.5297467926761514, P42=0.549998286759513, P43=0.6088248916549774, P44=0.5302935532204105, P45=0.5134392877761345, P46=0.5977226031217612, P47=0.7332536868901876, P48=0.7865965975677227, P49=0.7110014723264084, P50=0.6475643629855641, P51=0.6301160894188597, P52=4.968972018369859e-06, P53=1.7103285803049455e-05, P54=4.137686977824087e-05, P55=2.4394681373679654e-05, P56=1.564328842660914e-05, P57=1.1379090765385102e-05, P58=9.626286310126953e-06, P59=1.7307243675767134e-05, P60=3.78801083985424e-05, P61=0.000294736579949754, P62=0.013802857544100053, P63=0.0920794278747594, P64=0.13677015123683373, P65=0.09219371238851386, P66=0.08862025697289848, P67=0.11817869799881515, P68=0.0360136986572331, P69=0.002454376844469318, P70=0.001125133993668504, P71=0.0005123794704741393, P72=0.0006218780801453683, P73=0.0009178133226139034, P74=0.004762693510005101, P75=0.020437529678718144, P76=0.08271322157936747, P77=0.14968960081576194, P78=0.1516502334597335, P79=0.07399789693227832, P80=0.05299631250048374, P81=0.04875249624846398, P82=0.04771577491785857, P83=0.06277995629515917, P84=0.04753903696584381, P85=0.015010648002830612, P86=0.0033130623774675406, P87=0.03589804045287333, P88=0.1921198975984036, P89=0.09526960669364812, P90=0.02777860580774394, P91=0.012743287595134161, P92=0.006986260075319782, P93=0.005978639753628197, P94=0.009632254431446583, P95=0.012098818188866458, P96=0.014096552535415927, P97=0.0230726461265225, P98=0.02226210898811822, P99=0.02263297312355258, P100=0.005523139133157236, P101=0.0025397220423172473, P102=0.001169173055568854, P103=0.0033695023844030614, P104=0.008047162656805358, P105=0.021180598035989807, P106=0.018687821502733168, P107=0.002107641419516782, P108=0.0009325366642845875, P109=0.0033496195685140096, P110=0.002658174414580366, P111=0.005753140634014732, P112=0.004634232461067073, P113=0.003011970308807254, P114=0.0025417250591589164, P115=0.0018440555948386471, P116=0.0016054744165714854, P117=0.0005980125659416855, P118=0.001526903405776959, P119=0.0017461076219881459, P120=0.0006725950143516421, P121=0.001863720316268418, P122=0.003356732317243958, P123=0.001215640728971372, P124=0.0014078888766237765, P125=0.0016697872644739417, P126=0.0011267009883636323, P127=0.0011228941884147568, P128=0.001064141306322119, P129=0.001190783772028834, P130=0.0006061795920916538, P131=0.0003905810586031229, P132=0.00040210998570613493, P133=0.0002681454671414685, P134=0.00015667724554009607, P135=0.00024311281459482118, P136=0.0001525881665008539, P137=0.00013844885009349446, P138=0.00010042389203594404, P139=0.00012138525386507302, P140=9.426116397709423e-05, P141=0.00010655270035544912, P142=8.647621559554732e-05, P143=0.0002642024972873919, P144=0.0005120160665714822, P145=0.0005557164221081331, P146=0.000515121079820174, P147=0.0002720221304507363, P148=0.00017730860470603762, P149=0.00020198184994653904, P150=0.0002801463035061894, P151=0.00016915131497755828, P152=0.00021390768386114084, P153=0.00016587628032755712, P154=0.0001008496277072736, P155=6.635877678797344e-05, P156=2.647922971156245e-05, P157=2.294801153532479e-05, P158=3.812662982125321e-05, P159=0.00012017857188562273, P160=0.00018995532953452354, P161=0.00011562962214523436, P162=0.0001586759313819094, P163=0.0001777597279710271, P164=4.6161326153795765e-05, P165=2.3212325600011553e-05, P166=2.214008422431781e-05, P167=4.964727178408776e-05, P168=6.177747444407653e-05, P169=7.855493793787774e-05, P170=7.426950801904556e-05, P171=1.500679350242166e-05, P172=6.10805114551995e-06, P173=9.655062658444542e-06, P174=1.1374745964056959e-05, P175=2.2498074412231406e-05, P176=8.651468119930057e-05, P177=3.3721798880308536e-05, P178=1.639581291859343e-05, P179=1.93765907803461e-06),Row( P0=-587.7079261325939, P1=52.574279629057465, P2=-14.06774042284244, P3=-2.3196921990366386, P4=-16.804300616237246, P5=-15.027032780495098, P6=-4.804413552412491, P7=-4.01800357241878, P8=2.41744752672573, P9=-9.152590387638346, P10=-4.458922799992892, P11=-3.545699342708386, P12=-7.548523345428355, P13=-4.672978645249288, P14=4.211543310661213, P15=9.225121296554256, P16=12.797259758368554, P17=17.520599647599326, P18=12.54772386654805, P19=10.928663844847534, P20=-1.2594618428431363, P21=-2.7897173072966814, P22=1.2797016737920603, P23=-2.183124464607596, P24=2.186977569527598, P25=-4.571298582499386, P26=-10.296430176107739, P27=-6.819632038820971, P28=-6.229909987563392, P29=1.9894413197347418, P30=-0.21846415847271433, P31=1.0912757594055786, P32=-5.688901217677265, P33=2.8270351760451056, P34=1.667152987761056, P35=0.4001022045151192, P36=-4.362764067957612, P37=-3.9763334073000616, P38=-0.8207765556238508, P39=3.083651016798795, P40=0.7497672497657819, P41=0.5644761403609129, P42=0.42879386956030463, P43=0.4130916258917986, P44=0.43188178341168887, P45=0.42693998753573176, P46=0.4778020010944631, P47=0.559019468619131, P48=0.5671940380786868, P49=0.5996627860526671, P50=0.665200541986086, P51=0.6721716027724923, P52=5.448703143653509e-06, P53=1.742520332961324e-05, P54=3.553878516889453e-05, P55=4.293771419503765e-05, P56=6.308250800598476e-05, P57=7.682956953047262e-05, P58=4.0619758314338825e-05, P59=2.459443750433636e-05, P60=2.712982364325816e-05, P61=3.49589820150691e-05, P62=0.00047146595651785934, P63=0.006249376421345028, P64=0.02210803571519155, P65=0.13672898652901105, P66=0.46711756777390373, P67=1.0204068743665702, P68=0.4921673749348533, P69=0.032661734193849414, P70=0.007983271615484844, P71=0.0021535643063678336, P72=0.0008133237857397262, P73=0.0007542208571185313, P74=0.0008338296737433006, P75=0.0010309981588554032, P76=0.003821874940848992, P77=0.014327620886716338, P78=0.02159895082443552, P79=0.0282404061283622, P80=0.04137764928688634, P81=0.049382049561326094, P82=0.1080308608344697, P83=0.39666149480066004, P84=0.5368782292773939, P85=0.23274471444053962, P86=0.03047438736407495, P87=0.00817766363295121, P88=0.00513361335478157, P89=0.010596366393108911, P90=0.015775351191493554, P91=0.016265456918479887, P92=0.005266317956310554, P93=0.002290984783806319, P94=0.03604567728635186, P95=0.23014695778881072, P96=0.16263557788917524, P97=0.06784564519403918, P98=0.005889332283266131, P99=0.0006085630581246979, P100=0.0006316092610690144, P101=0.000835332671883667, P102=0.0010839833216130248, P103=0.016515635700062016, P104=0.07527784743123639, P105=0.08905118293227479, P106=0.08333861236204333, P107=0.0010720757107892218, P108=0.0004221134285284954, P109=0.0002840426493650802, P110=0.003092015709142289, P111=0.005809196730706562, P112=0.030225457421078596, P113=0.021933617376677435, P114=0.00010865002520130778, P115=0.0015474016507913968, P116=0.005022217381466415, P117=0.00892592816838607, P118=0.004236104650354879, P119=0.0006311692055366567, P120=0.0013404387655734278, P121=0.0042654896711912145, P122=0.005537404983711553, P123=0.0015335543863765882, P124=0.0002172045274084472, P125=0.000575524871976366, P126=0.0017153795461134016, P127=0.00045365257523479494, P128=0.0001229562549103675, P129=0.00038227910683354745, P130=0.0004892403601046727, P131=4.8647773053593094e-05, P132=0.000159979128233604, P133=0.00038225982530053334, P134=0.00011759721320771024, P135=0.00022395120856482807, P136=0.0004684106409604345, P137=0.0005155769372490709, P138=0.00020988415558431802, P139=0.00010508564346831356, P140=4.076845288727936e-05, P141=3.814251947647273e-05, P142=8.655061131584554e-05, P143=0.00012041199764428233, P144=0.00016870542793234842, P145=0.00019432972456125087, P146=0.00027002385511125685, P147=0.0001967330112957334, P148=0.00026634851371859275, P149=0.00023529972690359947, P150=8.940520813744924e-05, P151=6.844305199820199e-05, P152=8.501238464063563e-05, P153=9.314643222654048e-05, P154=0.0001258553272897794, P155=0.00016603907008232345, P156=0.0001887696494526511, P157=0.00024192647368593507, P158=0.00017570374958047002, P159=0.00016233678610821592, P160=0.00013959581180407964, P161=7.38509854733045e-05, P162=5.323701447735876e-05, P163=3.277963996200938e-05, P164=1.8422700164854308e-05, P165=2.5630206651517306e-05, P166=5.13327536557286e-05, P167=6.566589705371159e-05, P168=6.807363337466697e-05, P169=5.544522142759758e-05, P170=2.0028479593144365e-05, P171=1.85034184034771e-05, P172=4.258954022895453e-05, P173=4.005033635900939e-05, P174=5.036016430352695e-05, P175=7.55710297473027e-05, P176=0.00010571677695155144, P177=8.464159226253791e-05, P178=3.5082852642515044e-05, P179=2.9528008890475127e-06),Row( P0=-522.7083395092254, P1=38.51534282058627, P2=-24.456513503631857, P3=-2.5660535092094645, P4=-13.676847966815927, P5=-17.336593618863084, P6=-6.471364680370962, P7=-6.77712023138309, P8=-0.8665648117172738, P9=-9.086794637830875, P10=-5.276074667015393, P11=0.4752118845921041, P12=-2.168646329267819, P13=1.2631780457159023, P14=3.840033745073606, P15=4.767172091083461, P16=7.9658301675685035, P17=5.555186353335009, P18=6.133577705689368, P19=3.50412007535343, P20=2.3805014635093076, P21=-0.05391347750142105, P22=0.2642909413983845, P23=-0.353652211011546, P24=-2.70524579510079, P25=1.2177738025853515, P26=-6.361205624837003, P27=-2.232988977867756, P28=-3.4928605133331336, P29=-1.286248787054608, P30=-1.226407654288131, P31=-0.3625977604433998, P32=-1.468144235540808, P33=0.9478959732447674, P34=-1.1788424996235505, P35=-0.9227515276575959, P36=-0.048869164349153886, P37=-0.19044869575787046, P38=-3.520603783247318, P39=-1.640010929244304, P40=0.6068631388156689, P41=0.6057850565170197, P42=0.5931117762489307, P43=0.5753107938556318, P44=0.6244030620955633, P45=0.6429644702489254, P46=0.6483823198621393, P47=0.6462679256783866, P48=0.6715285138015121, P49=0.660289821602496, P50=0.6992965635370955, P51=0.6300738450690647, P52=8.226746922777906e-06, P53=2.274391676965951e-05, P54=5.738174539185571e-05, P55=9.429327810100593e-05, P56=0.00012793755962539957, P57=0.0001369773438859346, P58=6.648641800725384e-05, P59=4.734203154373077e-05, P60=8.872456748196387e-05, P61=0.0001862529198976866, P62=0.005451167560624275, P63=0.02710373935042363, P64=0.05469671613952599, P65=0.2819959557746363, P66=0.6412919403305382, P67=0.5099995224249627, P68=0.46258542778683015, P69=0.45109371725341985, P70=0.33252384908023164, P71=0.5106245340683745, P72=0.8476763418648233, P73=0.6735913609759617, P74=1.2774094692999942, P75=1.639514657219943, P76=0.6731751211545, P77=0.20676304409984833, P78=0.07674155377833229, P79=0.07227443935322035, P80=0.21704355579995388, P81=0.6426049031616052, P82=0.7805616219006563, P83=0.23796315683615601, P84=0.28236235238258706, P85=0.2734207860854939, P86=0.11992009568087539, P87=0.07236033264172174, P88=0.08245263077467123, P89=0.07510469286245823, P90=0.07502397912280559, P91=0.09487744362713198, P92=0.19120169631429698, P93=0.2869039216050859, P94=0.20174590576536908, P95=0.0903367557135804, P96=0.022039130185217918, P97=0.3457212301079428, P98=0.43887556998115745, P99=0.062338528869892704, P100=0.030035771252211553, P101=0.028068781919403755, P102=0.043002171171495594, P103=0.0287642506371914, P104=0.011271448057500598, P105=0.006987490609089636, P106=0.10009205786279715, P107=0.09068442020085789, P108=0.02553913891259867, P109=0.013390799465445767, P110=0.009111009207411374, P111=0.002444760625054847, P112=0.008834294516686592, P113=0.06614410582461257, P114=0.06979032034730472, P115=0.07810607435270567, P116=0.025501694027052935, P117=0.004660958811076406, P118=0.00925027505102679, P119=0.04669311498862764, P120=0.037595585232277726, P121=0.010160987405481579, P122=0.012129968853750148, P123=0.06220903161279629, P124=0.033216883621037196, P125=0.01337460929149711, P126=0.009678672360668815, P127=0.01071516273405225, P128=0.0069073306528282825, P129=0.007370656850173122, P130=0.00677311081597928, P131=0.002953245241025583, P132=0.0030512214763573965, P133=0.005139117087316711, P134=0.007009797763828966, P135=0.008541918711903594, P136=0.004470516160946411, P137=0.005439378951086899, P138=0.002310252897308465, P139=0.0013989701440313088, P140=0.0008839851432425833, P141=0.0015393427644162354, P142=0.0018104600082086274, P143=0.0018281934160626822, P144=0.003429848255501525, P145=0.0020982739161576943, P146=0.0018254671559044245, P147=0.0018823656721103019, P148=0.000938359730067277, P149=0.0005519161365681442, P150=0.00025573738512512203, P151=0.00032000455713679274, P152=0.00041939683000324506, P153=0.0006945509684947355, P154=0.0007525646527038638, P155=0.0005057314431819947, P156=0.0007818907018902085, P157=0.0008281312589644438, P158=0.00036132889590165886, P159=0.000286626545799955, P160=0.00028292175612414664, P161=0.00027659756151853795, P162=0.00028662364871231635, P163=0.00031260616207746874, P164=0.00016501588648297163, P165=0.0001544578883163286, P166=0.00017358730702781084, P167=9.537732580879966e-05, P168=0.00010645332205343376, P169=0.0001598112022208329, P170=0.0002215957911086778, P171=0.0001616254909579361, P172=0.00012691606074488753, P173=0.00017620920034319947, P174=0.00024184398074920388, P175=0.00027776889386956025, P176=0.00026636970006581654, P177=0.00017002649653263585, P178=7.719013909078202e-05, P179=6.861821683176907e-06),Row( P0=-531.1775953155474, P1=43.530817385314606, P2=-26.31521063931227, P3=-9.6283308220459, P4=-24.370455708234633, P5=-19.13784142148931, P6=-6.967452125335442, P7=-6.54463021352617, P8=-0.3384070906716768, P9=-10.088908703768313, P10=-1.6186771844857875, P11=10.13002591001019, P12=2.8272933601387025, P13=12.69825931456774, P14=11.359130313429542, P15=5.075126593048023, P16=5.849383040086681, P17=-1.5590074124856337, P18=0.06324533022482745, P19=3.1050587992901892, P20=2.284352482197321, P21=1.7794551904103117, P22=-0.2411186459383113, P23=-5.991827828857416, P24=-4.888618680288198, P25=1.111012282711511, P26=-1.4971964488149199, P27=2.4732620149353575, P28=-3.1675921786113985, P29=-0.3257033546269921, P30=-3.364980425268778, P31=0.18554753289207243, P32=-2.0929029376081894, P33=-1.0512999097080102, P34=-2.028694628821457, P35=-2.9783901896628624, P36=0.32584773704416015, P37=-1.011146905007681, P38=-0.2027124588247698, P39=1.218923814109989, P40=0.5744459783326518, P41=0.5887925931441652, P42=0.6157616962884748, P43=0.6422360524245185, P44=0.6768490225207303, P45=0.6069600897560045, P46=0.5427385189670405, P47=0.5126306320951775, P48=0.5995895119580482, P49=0.6713493466106608, P50=0.6679036857235037, P51=0.6002230430246092, P52=8.677151101520912e-06, P53=2.6772586747732897e-05, P54=5.385632737194379e-05, P55=6.554436720149318e-05, P56=5.804734999605264e-05, P57=5.453147940242897e-05, P58=3.90985196567567e-05, P59=3.9672827987925546e-05, P60=4.606135859829041e-05, P61=3.581699209313009e-05, P62=0.0005247160634418781, P63=0.009106812177786673, P64=0.033878021331299274, P65=0.06224695178623899, P66=0.1019685080016253, P67=0.18980695627348032, P68=0.33447957569254577, P69=0.7159281913390213, P70=1.1396134715067576, P71=1.9023379585499656, P72=1.3121550950591538, P73=0.953539267927157, P74=0.8631956434785271, P75=0.5234568563699109, P76=0.1527487655513412, P77=0.051286958512377336, P78=0.05934365413821258, P79=0.06399547883178726, P80=0.07245372148233477, P81=0.05652630346385687, P82=0.04507269119667404, P83=0.0564703303302874, P84=0.09340138336234076, P85=0.1269129127300247, P86=0.146838896825912, P87=0.3311219726920277, P88=0.5159886016773961, P89=0.6618230866927836, P90=0.9334605695530676, P91=1.1302682007056153, P92=0.6664695493565146, P93=0.19360250278302832, P94=0.06735746628380831, P95=0.03268633500649191, P96=0.019042729161113343, P97=0.04837208279029496, P98=0.060424648713878155, P99=0.23266649072499682, P100=0.5066970551487804, P101=0.6110675650198765, P102=0.16784256072584944, P103=0.048948112534649396, P104=0.020812670274478078, P105=0.01165544020588614, P106=0.010844981085843893, P107=0.011648028062958301, P108=0.028648849695289927, P109=0.037211194455777946, P110=0.034663284431715596, P111=0.011991606850342113, P112=0.01210502971564058, P113=0.014303363262063155, P114=0.010377001543425702, P115=0.04721257649723836, P116=0.05977782540127622, P117=0.05638076419128843, P118=0.022767778107144657, P119=0.01771163784935849, P120=0.012235872771126442, P121=0.029132970243011473, P122=0.052494750660663864, P123=0.0607958085792382, P124=0.059096535303819656, P125=0.04080247038586687, P126=0.014388322161951893, P127=0.016209835352593703, P128=0.023556376123874918, P129=0.02752778864989622, P130=0.005990948501582171, P131=0.007521224864375984, P132=0.006278562644796468, P133=0.0056770981618456656, P134=0.009428991128855589, P135=0.007742897567101167, P136=0.007054608706240883, P137=0.00914185968335188, P138=0.0011817647797077717, P139=0.0007323486920368091, P140=0.0007830590568048775, P141=0.0005836183960885379, P142=0.0007414663058668552, P143=0.0014234117845641626, P144=0.0018668095187175104, P145=0.0035997472737093164, P146=0.005528725799714093, P147=0.0064210773677325335, P148=0.006755606156733717, P149=0.003209112285602533, P150=0.0017667102672833139, P151=0.0007980644974891561, P152=0.0007236646842440868, P153=0.0004837647056240089, P154=0.0008724575121979097, P155=0.0006977649761773749, P156=0.000682872236444609, P157=0.000618436629890004, P158=0.0005117631417570356, P159=0.0002972555824699787, P160=0.0002514495820489021, P161=0.00012928536003063427, P162=9.341053005406324e-05, P163=0.0003309777761901422, P164=0.0003169106455132168, P165=0.0006535297322543984, P166=0.0007543939565236165, P167=0.00062000199609555, P168=0.0002151295887718526, P169=0.00012871591420369525, P170=0.00016453440004706335, P171=0.0002194274758169362, P172=0.0002797535081703861, P173=0.0002279193285839956, P174=0.0001392709776759295, P175=0.00030208499418225225, P176=0.0003445680337034797, P177=0.0005694412916108734, P178=0.0005838289651845741, P179=0.00010369127113032027),Row( P0=-607.3004892164774, P1=70.757670749173, P2=-22.85161556251188, P3=0.34947981274822243, P4=-11.878340545125637, P5=-12.866448665026617, P6=-4.098081039983265, P7=5.047081128165301, P8=2.5680338707707726, P9=-15.128580308156767, P10=-1.6382587585829453, P11=-1.1979472492180079, P12=-12.232737211005334, P13=0.4807533275118608, P14=0.4688648368053116, P15=-0.2985938816004672, P16=3.766685229102199, P17=2.789756166108205, P18=6.688734881017983, P19=7.506028649738335, P20=8.083610064050102, P21=6.802401693031476, P22=4.796411804742211, P23=1.0373403690393237, P24=-1.4075143809993786, P25=-1.3522724418834848, P26=-3.581569115492612, P27=-0.3625262496392152, P28=-2.5231336561669346, P29=-2.2607027754336375, P30=0.6192264462425392, P31=-1.7648780581713879, P32=-6.4174456347122995, P33=-1.5471620811078173, P34=-3.500673496143179, P35=1.7075394229164758, P36=-2.8559919969096397, P37=-3.514766711329716, P38=-4.974264200867198, P39=-0.4635366385039615, P40=0.6204804110477059, P41=0.5635259365691473, P42=0.575527981262726, P43=0.6095388374188248, P44=0.5202286449450828, P45=0.501294883141885, P46=0.5927692474460825, P47=0.6870951432100919, P48=0.696722565845858, P49=0.6186322239284917, P50=0.5965548168492054, P51=0.6223623696052081, P52=5.4244703956932175e-06, P53=1.5525481430923513e-05, P54=3.256421378170859e-05, P55=1.9541277281530552e-05, P56=1.3636889860591009e-05, P57=1.7977571207470247e-05, P58=7.715915874834975e-06, P59=3.101250662997718e-05, P60=0.0004809029924717216, P61=0.0017305153889386098, P62=0.017654799704591637, P63=0.06986865060011675, P64=0.06275440714065679, P65=0.06003893922966188, P66=0.17596434324562582, P67=0.2268610805646082, P68=0.07245649572474741, P69=0.0632992901709887, P70=0.15612084928232128, P71=0.08433947736775677, P72=0.00252534402203751, P73=0.0017247165986743768, P74=0.008435791682591719, P75=0.025204803498311508, P76=0.055541635640115536, P77=0.07069954424536809, P78=0.039166545331384854, P79=0.02874341292291636, P80=0.027175527663320296, P81=0.04206100983609135, P82=0.039994501608161405, P83=0.023100235767331754, P84=0.021245769128295566, P85=0.01316358099270928, P86=0.008077937516366932, P87=0.03765995905367936, P88=0.17409225124271252, P89=0.1714922282725784, P90=0.0420819889272093, P91=0.02053567505410663, P92=0.02152519904488941, P93=0.03808447359771193, P94=0.08093602587899329, P95=0.054397764744373196, P96=0.010935672817527964, P97=0.004447640912274999, P98=0.005445773448242789, P99=0.001262686386380679, P100=0.0016258925698012777, P101=0.00521843626052151, P102=0.027762141070034125, P103=0.032281783949307814, P104=0.022509778374136216, P105=0.002341262669662177, P106=0.0004528675342226847, P107=0.000499180344297478, P108=0.000992784828028475, P109=0.007179892364510952, P110=0.006677399378414314, P111=0.0025253922453152026, P112=0.0008777906202975478, P113=0.000795149694497687, P114=0.0006473731790602785, P115=0.0014005856054260118, P116=0.000923031015639371, P117=0.001672660785032211, P118=0.001609868262427996, P119=0.0012733403052408224, P120=0.001047320395029871, P121=0.0015305504598572047, P122=0.001596137699850149, P123=0.0026802987719992412, P124=0.0008842959224487983, P125=0.001073829490852978, P126=0.0015167438271459634, P127=0.0029855531937654054, P128=0.002792268441008539, P129=0.0008748245598681945, P130=0.0008609032095759494, P131=0.0005374743639893173, P132=0.00031052941050861786, P133=0.00027909278680658125, P134=0.00024737049830564296, P135=0.0002457030787410634, P136=0.0001173911643980102, P137=0.00019513553724933642, P138=0.00017588325453144914, P139=0.00016550294847753972, P140=0.0001288198037706731, P141=8.372273926828169e-05, P142=5.0509562672911655e-05, P143=8.832557956014829e-05, P144=0.0001904098267341907, P145=0.0006042401041316455, P146=0.0006908499679490716, P147=0.0005288958965861099, P148=0.0003511021718486064, P149=0.00039674245339369683, P150=0.00014388735307370722, P151=0.00018436371926961666, P152=0.00033794416928837587, P153=0.0002995681293061246, P154=0.00016578036478616232, P155=0.00027008467785693397, P156=0.00010654992852595957, P157=4.685525017091943e-05, P158=4.2436379546300894e-05, P159=9.200338266883527e-05, P160=0.00020107687483511606, P161=0.0002126535737060312, P162=0.00011938066660710818, P163=8.303710206542847e-05, P164=5.462713122622823e-05, P165=3.366091011322001e-05, P166=7.836607007071144e-05, P167=0.00020210439334692517, P168=0.00031046382797581885, P169=0.00020553245837270844, P170=0.00012084867650781066, P171=4.994466410415916e-05, P172=3.089032948659181e-05, P173=5.229721927331177e-05, P174=6.172680487261131e-05, P175=2.7167449127340708e-05, P176=2.3333546844806056e-05, P177=2.02050247710892e-05, P178=1.4782161705619744e-05, P179=2.721845380424811e-06),Row( P0=-516.0572186893521, P1=38.331723761028066, P2=-22.54965695570452, P3=-5.635044103721921, P4=-17.45027812288945, P5=-19.26249240625061, P6=-8.389881656623459, P7=-8.199830817234755, P8=-4.514130374180291, P9=-10.492558527596795, P10=-4.774202419239687, P11=6.963287401710067, P12=3.49997600838343, P13=10.450690473620243, P14=11.585304502222227, P15=3.103640445314171, P16=1.4975515104088946, P17=-6.072909278482798, P18=-2.5528509015138408, P19=1.9070813373110556, P20=-0.07748524002610992, P21=-0.8485551021256659, P22=-3.7375253189302793, P23=-3.7157643430007057, P24=-1.078582990242735, P25=1.7779245985635694, P26=-3.296425300430426, P27=-1.7734122920161384, P28=-4.568930035353222, P29=0.33437528662837884, P30=-1.5451418144388231, P31=0.3892127169769943, P32=-3.522015563553911, P33=1.0726137618913671, P34=-0.15092886480270834, P35=-1.3294939729030837, P36=-0.050423636700027336, P37=0.05610056312735888, P38=0.09177837205678031, P39=-0.13311278082882644, P40=0.634912204270937, P41=0.603220331694903, P42=0.6242027535757165, P43=0.6290285141320817, P44=0.6766031867640603, P45=0.6663158593018059, P46=0.6115395467958596, P47=0.603503216935486, P48=0.6010449260605378, P49=0.62411615658824, P50=0.6153180737554504, P51=0.6406388076300391, P52=6.2792288550682735e-06, P53=1.5194073701264449e-05, P54=5.364978186767138e-05, P55=0.00010713713899156536, P56=0.00013936946027403818, P57=0.00016408682433051368, P58=0.00011944776076809228, P59=0.00013549388886482063, P60=0.00017072545307141599, P61=0.00023491153242683583, P62=0.0005749265753103169, P63=0.003056329126238396, P64=0.008683952819411811, P65=0.07986628860047887, P66=0.3045840453251729, P67=0.37507191291155845, P68=0.3061721154586602, P69=0.6303613664469883, P70=0.801015359452868, P71=0.9567131939908442, P72=2.295045600195094, P73=3.5867319510455338, P74=2.0976980570038837, P75=0.761532792857498, P76=0.28110305668442115, P77=0.07455327490414106, P78=0.026397494950502365, P79=0.02136066986285948, P80=0.01966945690664172, P81=0.0160057292330858, P82=0.014851568582143424, P83=0.01759337184415391, P84=0.029930031615426997, P85=0.04329451584694623, P86=0.062366572401718015, P87=0.23724061487964, P88=0.46860812008371805, P89=0.26270434220884537, P90=0.4156159023499692, P91=0.8437307972708189, P92=0.6809587140971919, P93=0.2555555466462606, P94=0.08685296569945088, P95=0.052654917669736484, P96=0.029458132381891745, P97=0.028707399092931365, P98=0.015077490283056145, P99=0.013774697203500547, P100=0.1058389476113622, P101=0.15598533652584232, P102=0.09985532652248509, P103=0.267349025410826, P104=0.18368423650991186, P105=0.11701901012291169, P106=0.04855749774494123, P107=0.005470997323668279, P108=0.003774072823706022, P109=0.004374057452247709, P110=0.008737662049420388, P111=0.024071198355570585, P112=0.09162295781049337, P113=0.04019070642128317, P114=0.018090862419281945, P115=0.024890828494619675, P116=0.01759454970188909, P117=0.030573734578057003, P118=0.030397637787509348, P119=0.006675321610239585, P120=0.0065958478084211895, P121=0.030211724585900684, P122=0.09323036989485411, P123=0.037463602664385934, P124=0.004554142908170777, P125=0.009488708961212948, P126=0.024280220397987606, P127=0.023829075481859866, P128=0.00819857842775136, P129=0.004318546727086022, P130=0.004659411170062187, P131=0.005144462064451284, P132=0.0033171109884856495, P133=0.003647797277080131, P134=0.0036559745481287635, P135=0.0019555467377370167, P136=0.0014602055392459086, P137=0.0017617722150303306, P138=0.0009443358076773823, P139=0.0006562487412252768, P140=0.0006917139974262799, P141=0.0006904479905750681, P142=0.0012042689801979533, P143=0.0017977426189098344, P144=0.0018450878768843445, P145=0.002634497065192742, P146=0.0033301679108476336, P147=0.002216337416770154, P148=0.001770794999996205, P149=0.0008212066335623367, P150=0.00041034135182410565, P151=0.00028717213848125975, P152=0.0007652267494176702, P153=0.000648689358892246, P154=0.0007288585658562887, P155=0.0007529367711499641, P156=0.0008151319399877477, P157=0.0004300433356909684, P158=0.0006481496613618397, P159=0.00034910501176413014, P160=0.00037326453537290415, P161=0.0003995676909413031, P162=0.0005168443122381434, P163=0.0006485722698734459, P164=0.00145759801720223, P165=0.0006239159453135002, P166=0.00038908977044922917, P167=0.0003094556073175376, P168=0.0002865703735269183, P169=0.0002911134014877388, P170=0.0002547015974956104, P171=0.00013804437471596578, P172=0.00021093632683980484, P173=0.00020058180876283223, P174=0.00016735292118415565, P175=0.00017843969939815632, P176=0.0001995349742101736, P177=0.00019150455490599025, P178=0.00010121153310712773, P179=1.0311004635779904e-05),Row( P0=-474.3134657613738, P1=26.27248276944662, P2=-28.09016458801958, P3=-9.681184665752214, P4=-23.36899544739875, P5=-20.681713086232122, P6=-9.506008021276049, P7=-7.877078564488139, P8=-0.9966491776965416, P9=-5.042520982959208, P10=1.2627364324130035, P11=14.625907680653203, P12=7.992417245242568, P13=8.506672195987855, P14=10.334609238991803, P15=-3.0313177782244662, P16=-0.16984535908332532, P17=-3.2139723150564805, P18=2.0190841379053226, P19=4.056762246574193, P20=-6.399326301565007, P21=-1.1096692668086927, P22=-1.885443951234039, P23=0.13506885795769696, P24=1.678881565785398, P25=-0.8186143562733119, P26=-5.355808124526817, P27=-1.0325287604335907, P28=-0.7304802901912826, P29=0.8057830693494659, P30=-1.8491584980474713, P31=1.2203047928670017, P32=0.6036870154571249, P33=2.8832871250119965, P34=-0.04104263817835183, P35=0.9193086253624144, P36=1.4480328226360433, P37=-1.610474664236661, P38=-3.1433905611938, P39=1.6865095632320477, P40=0.6212843648664728, P41=0.5374888024764667, P42=0.506629068380739, P43=0.49246859619573224, P44=0.4868618967706142, P45=0.4852587643537951, P46=0.5340791640050467, P47=0.5856520447932697, P48=0.5384169156594844, P49=0.524508755799299, P50=0.5597900413499791, P51=0.6252039348349197, P52=9.171130153326752e-05, P53=0.00010289890494931111, P54=0.00012613307652767805, P55=0.00020744158766045865, P56=0.00024371457105071994, P57=0.00028702123662882866, P58=0.00018746043538860372, P59=0.00015074154666917656, P60=0.00018911894169606366, P61=0.00020378273775658812, P62=0.00021167998270319162, P63=0.00015489872252435593, P64=0.00021825059298754028, P65=0.004202503802372285, P66=0.06539444297084615, P67=0.2169981202618765, P68=0.17313829046419218, P69=0.23312234389497832, P70=0.6927613612981679, P71=0.8068163063687117, P72=0.7531748427018827, P73=0.8761781838663503, P74=4.010977146120419, P75=7.230738190935386, P76=3.3977555352066244, P77=1.5729886522918302, P78=0.597082080829089, P79=0.21156464389503513, P80=0.08189497257683996, P81=0.04132927417099147, P82=0.03798549272644315, P83=0.0744563828459434, P84=0.09771964318326508, P85=0.0622067928578697, P86=0.02534533872766468, P87=0.06602907353778258, P88=0.3840608386215309, P89=0.729864143294335, P90=0.5441072533495872, P91=0.4918588360639997, P92=0.8828127722054465, P93=0.6112547358406173, P94=1.1002913328726258, P95=2.3933983781800716, P96=2.4965521585503585, P97=0.7331779593897128, P98=0.10475018175259042, P99=0.007974408333721511, P100=0.029125776118890886, P101=0.1784604671489384, P102=0.08415390351855168, P103=0.028182297527750037, P104=0.05365245893673991, P105=0.12437480737384261, P106=0.12273648123995405, P107=0.23499763590130576, P108=0.9013743538046582, P109=0.6120679729018561, P110=0.02832847148072519, P111=0.007848648809379147, P112=0.008547071502527677, P113=0.03378750611452569, P114=0.053185030807439325, P115=0.028061110533358222, P116=0.017650086617652317, P117=0.011851862835287247, P118=0.016048223267748074, P119=0.009559995287497496, P120=0.010733730867346584, P121=0.02845911044140691, P122=0.03538317352571642, P123=0.05706776490483416, P124=0.05819037580906036, P125=0.1078968388808248, P126=0.03422682820074059, P127=0.0047745148765971305, P128=0.004991252270581834, P129=0.04605875873754409, P130=0.01810185860656931, P131=0.0030199116299931203, P132=0.0031178200799380786, P133=0.004668125925874771, P134=0.006850864879350445, P135=0.009070061776459751, P136=0.009246181078963593, P137=0.006400143325294124, P138=0.006705458597466456, P139=0.003954577687019518, P140=0.002399033974113677, P141=0.0020973159297943867, P142=0.0031705166435990386, P143=0.0030281922198047142, P144=0.007360122747251578, P145=0.016988076096725434, P146=0.011073288640211886, P147=0.004201859035791232, P148=0.003151010232818824, P149=0.0042885541326717624, P150=0.003440837838346297, P151=0.001919023177584397, P152=0.0015028607640805308, P153=0.0014847164345762954, P154=0.0008958115864800625, P155=0.0007084590120731602, P156=0.0007882581332078572, P157=0.0010567291869301657, P158=0.0008992719907673194, P159=0.0014630667949218784, P160=0.002409253123919843, P161=0.0025018024344541836, P162=0.002593634739759382, P163=0.002697826996980707, P164=0.0015171823580977422, P165=0.0008795626667663811, P166=0.0014480894391761937, P167=0.0017578857888481722, P168=0.0019698163674302938, P169=0.0014682243404451302, P170=0.0007261966971583931, P171=0.0007231611750909685, P172=0.0008166633364395571, P173=0.0008365286033930961, P174=0.0006294060279822922, P175=0.0008040747631540183, P176=0.0007341848108559339, P177=0.0005819180945055644, P178=0.0004302536149849969, P179=4.8101138163661866e-05),Row( P0=-420.67862613646673, P1=19.64737652879709, P2=-38.282066354781925, P3=-8.895069439743109, P4=-24.07840109878418, P5=-27.276601606280476, P6=-5.771423180847648, P7=-1.1875909749838782, P8=9.78113246721576, P9=6.071431944537465, P10=4.109396580138666, P11=10.133642756042208, P12=-10.570260363738027, P13=-9.257332462319354, P14=0.2598424687753616, P15=0.01559155946038634, P16=7.781938875611173, P17=2.483618749073827, P18=0.49066228816689755, P19=0.8129769509854466, P20=-3.4217249807514363, P21=0.11241802501451863, P22=-4.940622278658609, P23=-6.041157819267409, P24=-1.6725827371458548, P25=4.800316754491557, P26=-4.676522045498165, P27=1.4128038151333882, P28=-1.2975752609694355, P29=2.066197060390907, P30=-1.374163574474993, P31=-0.1630114890583709, P32=-0.10100138345351839, P33=0.5451334218691197, P34=-1.5265783422954267, P35=-1.9224996053446028, P36=-1.1436993260284452, P37=-1.9466439072650987, P38=-2.7678461046545686, P39=-0.549378311322159, P40=0.7110227163890209, P41=0.5534234076665521, P42=0.5004612114146958, P43=0.462631727989202, P44=0.4650391444893342, P45=0.47193645456542255, P46=0.49352097066702055, P47=0.5152928393545597, P48=0.5521887256835453, P49=0.6531804741280272, P50=0.7385511693200238, P51=0.7872500719562595, P52=2.3263006512491075e-05, P53=5.364290519333938e-05, P54=0.0002547559686195485, P55=0.00047604253467001825, P56=0.00047381400970194907, P57=0.0004415116838543454, P58=0.000308006226789241, P59=0.00027729506428867464, P60=0.0005193061500060527, P61=0.0006142318631141118, P62=0.0008875217612582792, P63=0.0010127422335870038, P64=0.0007454707643507828, P65=0.001365408753549883, P66=0.012148758806859401, P67=0.043803019645928455, P68=0.08945378168515887, P69=0.220753722896718, P70=0.23713862115982434, P71=0.126622321643824, P72=0.16636147970544884, P73=0.15801477538943595, P74=0.22359213330982436, P75=0.3673906025566058, P76=0.7455871552404667, P77=3.3230056050773675, P78=8.40924968558421, P79=13.21163611862785, P80=19.8177123856462, P81=19.587714103925855, P82=22.541109181754436, P83=24.277425570031607, P84=17.78636775552876, P85=4.884239771458892, P86=0.4273380094091116, P87=0.19095054449271642, P88=0.12760319690609911, P89=0.07092858924470632, P90=0.07889961601943525, P91=0.15622125496658323, P92=0.1889410265700721, P93=0.15048503701413812, P94=0.26978417214617956, P95=0.37311181761347534, P96=0.3911845023199131, P97=1.5535149622672022, P98=2.09211877615253, P99=0.7419022314623221, P100=1.3948728256681289, P101=2.497527650114249, P102=2.703731756641618, P103=2.5541374314002234, P104=0.852078814150103, P105=6.492568927134708, P106=2.408783982946461, P107=0.04092486086217968, P108=0.029344818274884197, P109=0.03173628491527045, P110=0.07339134063549746, P111=0.16291171418529554, P112=0.15638370093854145, P113=0.24274822462065943, P114=0.6111582840875368, P115=0.6761835239228247, P116=0.9296622211732091, P117=0.3585917102162839, P118=0.06347580462317393, P119=0.1909356871952609, P120=0.2530866726626528, P121=0.34538187374391494, P122=0.32299287949381855, P123=0.11851406401513045, P124=0.06831355591512427, P125=0.17344434031161346, P126=0.1539220751677774, P127=0.056922009231840646, P128=0.06715083419039279, P129=0.05675538340735525, P130=0.027726459002325923, P131=0.03110815478128824, P132=0.03162623183593598, P133=0.020723479241318097, P134=0.03640153263292521, P135=0.04663885455021787, P136=0.027455599657013753, P137=0.02178877060882686, P138=0.013281376716554865, P139=0.014632888466455989, P140=0.008742348633272326, P141=0.009638286493387709, P142=0.012654332118556355, P143=0.03750570420033014, P144=0.06802855398372273, P145=0.05162046242045799, P146=0.057093560035643975, P147=0.032315625497167315, P148=0.011727299066747753, P149=0.009182218565732876, P150=0.008647986055576868, P151=0.005723800954771202, P152=0.003091028839301538, P153=0.004838243851906355, P154=0.0069779243852028695, P155=0.004212541664634554, P156=0.0042513036759746865, P157=0.006307729230495974, P158=0.004594074012913542, P159=0.0037030063603349407, P160=0.0032140287340601706, P161=0.0027077321877432485, P162=0.002364290777598794, P163=0.002802958514958332, P164=0.006915664089184056, P165=0.006148249468923946, P166=0.004030639231016016, P167=0.0014886215205832372, P168=0.001090360896918859, P169=0.0009510955373546877, P170=0.001375489835038293, P171=0.0018613685690531863, P172=0.0017172777174288912, P173=0.0016768782259243666, P174=0.0013175613986172962, P175=0.001676157483243975, P176=0.002244739767502595, P177=0.0018235050343691033, P178=0.0006676091306781627, P179=7.375358534474117e-05),Row( P0=-385.01071157146856, P1=9.51227226955664, P2=-35.83170935309367, P3=-11.077395445316746, P4=-26.27847816083589, P5=-22.684427375156613, P6=-6.9384252386099385, P7=-0.7124308936467906, P8=11.304774376748385, P9=9.701384935829031, P10=2.090583146206163, P11=5.011232155624997, P12=-12.771004586643711, P13=-9.020216884634893, P14=4.3707299587856, P15=3.982561265195709, P16=4.823640511880097, P17=-3.8366862190857125, P18=-4.310112648095732, P19=0.5567278564472765, P20=-4.4568874071578755, P21=0.1883813270387827, P22=-4.182847220617402, P23=0.1976484720235337, P24=4.5928821186412545, P25=0.10136945731556257, P26=-6.81542086510555, P27=1.0656272636326145, P28=1.2067672288064084, P29=1.3559951540332515, P30=-0.9942946432456378, P31=2.1558221971983675, P32=0.34950485822450733, P33=-1.1685976904310578, P34=-1.9987521787701679, P35=-1.063269261558341, P36=-4.680555296683577, P37=-1.9042769224951561, P38=1.5813401368487672, P39=0.7281051990971507, P40=0.7414212589519877, P41=0.6586918837347722, P42=0.5162292752044799, P43=0.44981038103980214, P44=0.4601066938027325, P45=0.46039535949170024, P46=0.4748368985350737, P47=0.5226012497145343, P48=0.5637681943701541, P49=0.6028468220710785, P50=0.6816669669240354, P51=0.7099208148924211, P52=5.4891480056105565e-05, P53=0.00019444604360628745, P54=0.00044405520174317734, P55=0.0005852625304820528, P56=0.0009105245939264382, P57=0.001594751517458099, P58=0.0012099739261632711, P59=0.0014101444172325251, P60=0.0033871279560459024, P61=0.010120116150021322, P62=0.017577074732241352, P63=0.011549242990368568, P64=0.0034602068422093847, P65=0.004536104443619655, P66=0.006461108467575811, P67=0.01192276673392258, P68=0.04088008164865299, P69=0.050038900224672854, P70=0.051845546065217324, P71=0.0729539767433167, P72=0.08779143967942357, P73=0.08879581051017446, P74=0.2849387229319526, P75=0.3949138245610533, P76=0.5047966054267387, P77=1.6389831862606428, P78=1.9395922102648857, P79=2.5150859225010316, P80=7.017716245694653, P81=14.45702937382256, P82=23.667763230025585, P83=59.08444345690215, P84=81.82687221484866, P85=44.60473086105143, P86=14.642879890535413, P87=7.4180579109495355, P88=1.116632431907797, P89=0.13383291189868035, P90=0.125253912130532, P91=0.1519342323763509, P92=0.1050471676819567, P93=0.06817240485181322, P94=0.0899382074217428, P95=0.09335569084309055, P96=0.0910322405551095, P97=0.34361538906323963, P98=0.4002534496865231, P99=0.1390385420224785, P100=0.1251676901860834, P101=0.25865057022788396, P102=1.6928968796071282, P103=3.5361230178150937, P104=3.688193354006362, P105=1.7638895941622685, P106=1.1057405735177492, P107=0.6089203665076578, P108=0.7509940198183892, P109=0.13808903516248947, P110=0.10182258242237818, P111=0.06158658695607283, P112=0.04779997041712288, P113=0.05557900115684075, P114=0.2508362045217184, P115=1.1662806073611836, P116=1.3422496822166694, P117=1.4309259339756328, P118=0.6791772866562416, P119=0.22096603933218262, P120=0.11431438489200624, P121=0.06513158033629887, P122=0.05094006017191582, P123=0.193174203826013, P124=0.2090411343292606, P125=0.8666048127368583, P126=3.1359903385059513, P127=2.4482393377066707, P128=0.09060062171205263, P129=0.011742912710272129, P130=0.007038459580633315, P131=0.03336861369204786, P132=0.3299460293161314, P133=0.2361868564721664, P134=0.061484838955778595, P135=0.022813790585449304, P136=0.07883071714071596, P137=0.12973864432960835, P138=0.29493926439521856, P139=0.29419561327131966, P140=0.03214411339055499, P141=0.01559107506784714, P142=0.044815975872472866, P143=0.11871203748481227, P144=0.08618790730083269, P145=0.051030118349114174, P146=0.13838591819033505, P147=0.3496131840842682, P148=0.14815423414541937, P149=0.014253788776839879, P150=0.007895528968321045, P151=0.012999781996183993, P152=0.005992737049526232, P153=0.004180352826940242, P154=0.004720032641290145, P155=0.004823664244167868, P156=0.006110544199355334, P157=0.005059654177201964, P158=0.005633342705551174, P159=0.007024950517941169, P160=0.00760116095063502, P161=0.006463351672168534, P162=0.00686775494637945, P163=0.006501157482232573, P164=0.008043880942290086, P165=0.01154731329259522, P166=0.007927220378863219, P167=0.0047127696341749134, P168=0.003503622321109323, P169=0.002558899231249953, P170=0.0021386986682468026, P171=0.003076349323518921, P172=0.0032715104911276993, P173=0.003563958713545575, P174=0.00320459833246642, P175=0.0026530683185923627, P176=0.0023980946331483804, P177=0.0030453339259410342, P178=0.001801322178170086, P179=0.0003055384193575481),Row( P0=-561.7476772446664, P1=46.393810261565385, P2=-14.95046016366268, P3=-7.630080832293456, P4=-20.2733589715356, P5=-16.603766463358802, P6=-2.53405683609689, P7=-5.21803619690638, P8=0.8803924037505706, P9=-8.84002308477815, P10=-3.7812686387046965, P11=-2.0147118669233786, P12=-8.37009009646978, P13=-4.33672895436884, P14=2.77865968706892, P15=6.840616680900321, P16=15.59110911566275, P17=16.634767717020935, P18=11.665651013096928, P19=8.717532855875968, P20=2.6425949339595665, P21=-2.137846223223695, P22=0.3254212877336521, P23=-4.048334179485332, P24=-2.968683829028923, P25=-3.4445466793043336, P26=-8.194889535265053, P27=-2.77672744933013, P28=-4.372395729431766, P29=-1.5530184249892145, P30=-4.016511613686862, P31=2.130151127888562, P32=-2.463847427240887, P33=3.524168570686682, P34=-2.199389197505395, P35=-3.75148272995811, P36=-1.5456697841915088, P37=2.816639900116531, P38=0.7015449604538244, P39=-0.2732130666579087, P40=0.6490614567744731, P41=0.5978018289414015, P42=0.47476997648193925, P43=0.3886030542145419, P44=0.3854689124982283, P45=0.449823290319728, P46=0.41305552338938795, P47=0.41651807512763706, P48=0.5140893485309828, P49=0.5984389129277221, P50=0.7045344737891932, P51=0.6758004555291255, P52=6.212884055941653e-06, P53=2.250901672765621e-05, P54=4.494902688786565e-05, P55=4.9063813899463506e-05, P56=5.292677691578492e-05, P57=3.209443602388082e-05, P58=1.5331788737194355e-05, P59=1.8101281718249872e-05, P60=2.490565972100939e-05, P61=3.696583682805244e-05, P62=0.0001920155392669278, P63=0.003057006354980134, P64=0.04198694222416072, P65=0.3386810840577122, P66=0.7047295180033781, P67=0.867817174748633, P68=0.914088010427702, P69=0.48135940235149655, P70=0.08313858287748747, P71=0.008584690998788929, P72=0.002233338927077564, P73=0.0016696247485483678, P74=0.0012495751798096816, P75=0.0013270252213656495, P76=0.0010565249033369748, P77=0.00529921206783957, P78=0.02172096350835407, P79=0.05975599592113122, P80=0.13746400877437806, P81=0.1118831219708746, P82=0.17344290162249848, P83=0.4129439601143104, P84=0.4056355332403694, P85=0.4270979201206878, P86=0.22435243556919665, P87=0.050348750078357464, P88=0.007230136430118906, P89=0.0027825170272913675, P90=0.010540871953257525, P91=0.03135154823624508, P92=0.07250705856394786, P93=0.04811158944386389, P94=0.04546065569036068, P95=0.05259927126167262, P96=0.025659602417542643, P97=0.1830270199333899, P98=0.14371201868365413, P99=0.016081951517297195, P100=0.014557660447676247, P101=0.038490741102174646, P102=0.01258430103815383, P103=0.005875653741674871, P104=0.008660253136542225, P105=0.005424632522185647, P106=0.02023995544402736, P107=0.006308930405635585, P108=0.006056473203347986, P109=0.002506325801006555, P110=0.004973958162489305, P111=0.004899187030271989, P112=0.0034806005290576028, P113=0.019254316746525037, P114=0.007188719376715955, P115=0.003078220916373089, P116=0.0026317068587130106, P117=0.0009893227933371925, P118=0.019084709883901072, P119=0.022966807773635516, P120=0.0023865872793850523, P121=0.0029232831774801383, P122=0.0021665008270849917, P123=0.02879798102804241, P124=0.015216368340531406, P125=0.0015001201883889604, P126=0.0009728505635825582, P127=0.0034079450076588366, P128=0.004053200118222178, P129=0.001990597207673421, P130=0.0004621043745120572, P131=0.001733194603391756, P132=0.0013272386467386731, P133=0.00020343460809989228, P134=0.0015118722406161362, P135=0.0018353725958015071, P136=0.0003228771553873654, P137=0.00033229505953968837, P138=0.00015046855047583962, P139=6.870928807697389e-05, P140=7.144486741750103e-05, P141=7.178538424679715e-05, P142=0.0001502540705615539, P143=0.0003047841019439461, P144=0.0006087917454820804, P145=0.0007322587785964558, P146=0.0006306624285249297, P147=0.0010147709398610392, P148=0.001416730963022643, P149=0.0008675574227296047, P150=0.0005162438435113422, P151=0.0005366796948279046, P152=0.0005031867632276353, P153=0.0003967963867928216, P154=0.00024144679694716715, P155=0.0002491021500907507, P156=0.00037288239445223455, P157=0.00038436792098359217, P158=0.0004579995796702742, P159=0.0005411526220792008, P160=0.0006938683227006423, P161=0.0004757218479974785, P162=0.0004322047438749735, P163=0.00021644294424215415, P164=8.30376848530655e-05, P165=0.0002496289907730503, P166=0.0003874514883258122, P167=0.000500682566583939, P168=0.0005092816852547084, P169=0.00029988240340361467, P170=8.623208907665432e-05, P171=7.120776497363372e-05, P172=0.00016223736033664572, P173=0.00021488713658557708, P174=0.00025308559511867494, P175=0.00020530402380194594, P176=0.0002344809282375083, P177=0.00023877327872959814, P178=0.00011270738522530168, P179=2.1530165124823068e-05),Row( P0=-461.10348877028156, P1=27.464057438422174, P2=-33.04689514049341, P3=-9.070250967716431, P4=-24.203973733249054, P5=-20.834389717433673, P6=-7.598446252966045, P7=-6.818587285953757, P8=-3.8132788093168823, P9=-8.20085961275979, P10=-2.282105120728292, P11=10.096649247403345, P12=2.497500975178411, P13=7.875816823121628, P14=8.705626386323807, P15=-2.400126611056834, P16=0.6045930249601028, P17=-3.749136645700756, P18=-0.6961886904992977, P19=4.372585911331664, P20=-2.77857618548018, P21=0.6155155241682253, P22=-3.0244105229298555, P23=-2.3825138291977166, P24=0.6007340331209232, P25=-0.3095337456588473, P26=-3.6390288470257097, P27=-0.05360616010845088, P28=-3.192491802490907, P29=0.3049312011604635, P30=-2.5015552343608807, P31=1.0066313442899344, P32=-2.1215976027933716, P33=0.2877151320230451, P34=0.18285182244353423, P35=-0.14703704438448909, P36=0.15633335586542088, P37=-0.7587762800077235, P38=-1.5206264677450805, P39=-0.16403866814334842, P40=0.6061703214354596, P41=0.565904989674687, P42=0.578269558244534, P43=0.6028039160548203, P44=0.6438936630168018, P45=0.6429712640982557, P46=0.6585584740761593, P47=0.6312057767111285, P48=0.5928701844740372, P49=0.5839030387935996, P50=0.6688110285418876, P51=0.7024961830540584, P52=7.623786555903782e-06, P53=1.766724352061782e-05, P54=8.926403580200164e-05, P55=0.00020096067740835847, P56=0.00023404498187278684, P57=0.00020862649983481585, P58=0.00011960818694941569, P59=9.722472937368745e-05, P60=0.00012555498145567657, P61=0.00017465752815783907, P62=0.0004874051821546608, P63=0.0011920844407159598, P64=0.002479410725980743, P65=0.014095611596050151, P66=0.12428824597170234, P67=0.32167725222733323, P68=0.24743525370298775, P69=0.4248079119351971, P70=0.6017386956092797, P71=0.715871141362385, P72=2.0684130712629893, P73=3.649263159364562, P74=3.2217820084234288, P75=2.4962438731602994, P76=1.6405881357293126, P77=4.297349733688692, P78=2.8070320129485635, P79=0.5084737676609891, P80=0.05739794005176522, P81=0.03040841958814708, P82=0.0396392024652107, P83=0.056320313333724954, P84=0.0828757016102069, P85=0.08208867928319544, P86=0.06913791892172423, P87=0.22119031300877487, P88=0.43254394548834485, P89=0.5829130540963013, P90=0.7734536100896267, P91=0.9039420899940597, P92=0.720775680045682, P93=0.514153207733942, P94=0.8139433069391193, P95=1.5642589634939374, P96=0.7410867024376, P97=0.12352411944199324, P98=0.0879731264995007, P99=0.30153184861572047, P100=0.482801088796074, P101=0.26519395799521434, P102=0.5514442111674064, P103=0.23019172217866202, P104=0.15443176476638645, P105=0.13142227386319189, P106=0.1608355445829724, P107=0.19062336350429626, P108=0.1573423747260494, P109=0.023597895807756814, P110=0.01478437441808427, P111=0.05062413780022545, P112=0.05788514583331923, P113=0.025555829976649437, P114=0.015320783425075462, P115=0.0456401096543906, P116=0.05524267537902378, P117=0.016537352772939495, P118=0.01570811967169342, P119=0.02110670290783477, P120=0.015248476763394503, P121=0.10120759789905935, P122=0.060125564707512916, P123=0.025367181993911293, P124=0.012934452564194756, P125=0.014206364367062027, P126=0.045034455677498204, P127=0.027727275992762413, P128=0.010881902697307658, P129=0.011810339017139227, P130=0.0122084386227753, P131=0.003921755709165296, P132=0.0059509446613904395, P133=0.016162040439247318, P134=0.02240947039489003, P135=0.012247917575163914, P136=0.008754273826764466, P137=0.007282523222756087, P138=0.002474223776531875, P139=0.0015085649990786943, P140=0.0017404184708165653, P141=0.0060657840841424095, P142=0.008595859280082626, P143=0.0034065464134840373, P144=0.008785612009896387, P145=0.013267882899200126, P146=0.008117008253852821, P147=0.00723395165867688, P148=0.004134221015914257, P149=0.0036599397013687795, P150=0.004290598733665674, P151=0.002980628221252093, P152=0.001960049568302987, P153=0.0014251863815855509, P154=0.0012449471136074888, P155=0.0014031102331521796, P156=0.0012926137438311893, P157=0.002223528614097988, P158=0.003983534316669885, P159=0.0032951668329313046, P160=0.0036723957404970753, P161=0.0024452225204440246, P162=0.0017768432673540844, P163=0.0011945648363302793, P164=0.0011509767138029815, P165=0.0011773980828838524, P166=0.0011786845846596331, P167=0.0010925588905924834, P168=0.0009133855250947647, P169=0.0005545327557258767, P170=0.00038802500254371503, P171=0.0005042877352632782, P172=0.0005018647947267355, P173=0.0004672217042843299, P174=0.0004736095823357613, P175=0.0004947246809160643, P176=0.0005209580196301071, P177=0.000556372019418682, P178=0.00027390129284190535, P179=4.651110300633335e-05),Row( P0=-556.1138625145045, P1=52.0789792594029, P2=-21.639267678259188, P3=-1.1294699593735025, P4=-26.21451403387661, P5=-13.883605801004416, P6=-2.613155002148682, P7=0.6549368521987089, P8=7.5729568293903435, P9=-6.1909325239660085, P10=-6.520506682624677, P11=-6.109573606336034, P12=-0.38575658459803935, P13=-8.974487066936454, P14=3.439663691198211, P15=9.341280498491205, P16=5.89516985107045, P17=13.19444741255529, P18=7.480345330503503, P19=7.727757771071756, P20=4.062829926444322, P21=1.7592014935022389, P22=-1.522752628526445, P23=-4.109299940150198, P24=-3.617655018857871, P25=-1.5166030105559782, P26=-7.4612453395637734, P27=-3.89052093507813, P28=-6.169727485433313, P29=-1.459081741103941, P30=-0.3825548370716018, P31=-0.3324237810133108, P32=-4.875600622070794, P33=0.7040231316406678, P34=-4.567766731985714, P35=0.5649125954423606, P36=-4.585169662579855, P37=-1.438336520737552, P38=-2.1565391634178135, P39=0.25730974217976393, P40=0.6459737709091932, P41=0.5718866006290086, P42=0.5102709789886742, P43=0.5451115187768387, P44=0.5556219759446166, P45=0.5608628315824853, P46=0.5697280302194692, P47=0.5949764939408345, P48=0.6401392524207963, P49=0.6396287084074701, P50=0.6604816179061452, P51=0.6563952372413224, P52=7.069855789992548e-06, P53=2.25795695584177e-05, P54=5.775926466755145e-05, P55=7.484382067137241e-05, P56=9.989809674903218e-05, P57=8.599610629535053e-05, P58=3.5274527988205634e-05, P59=2.122402489633882e-05, P60=3.922330338987811e-05, P61=6.485762343327505e-05, P62=0.0013505974031863262, P63=0.02719250760660312, P64=0.11596951903580467, P65=0.20079725018890396, P66=0.4027934923249116, P67=0.42864861058857595, P68=0.20422830540835515, P69=0.11328381342000736, P70=0.0958734937793169, P71=0.08697292631936517, P72=0.014386737970688571, P73=0.0055150990812900345, P74=0.002207028623067643, P75=0.001938300598477299, P76=0.012419009942875186, P77=0.07666514556365614, P78=0.050718139751295084, P79=0.02583484762868213, P80=0.05549011907486865, P81=0.06686603828727175, P82=0.08154514543284752, P83=0.1113012093591208, P84=0.16091836830360048, P85=0.1532980872395721, P86=0.11544050670013004, P87=0.03414544544314355, P88=0.01594824100563285, P89=0.049721980142363544, P90=0.030964440543336566, P91=0.013219376114454388, P92=0.029301148276086257, P93=0.04236767088836212, P94=0.09183728641334225, P95=0.09753977011668075, P96=0.014041004198220713, P97=0.005775403757859241, P98=0.00612850683885835, P99=0.001386286857078913, P100=0.001290155941179826, P101=0.007576776420171558, P102=0.011550492960072139, P103=0.048173593960349106, P104=0.07240773658022745, P105=0.006139858135339014, P106=0.0024455204328630883, P107=0.0009367493450137842, P108=0.0028509378542367694, P109=0.0016298783880786423, P110=0.0030645504138722725, P111=0.003917143017112827, P112=0.0011624652379834278, P113=0.0013772555650763973, P114=0.001992584389996454, P115=0.0016277870685904236, P116=0.001757358826322268, P117=0.0011091228119212292, P118=0.0017573712753917725, P119=0.0043415311840414444, P120=0.003307478218784379, P121=0.0022156285574257513, P122=0.0014524477495601628, P123=0.004740184192455895, P124=0.004225332346290201, P125=0.002643854454249562, P126=0.0005284775686168435, P127=0.0018645013819720002, P128=0.0026112211903174605, P129=0.0006484566277925294, P130=0.00016492977993038755, P131=0.00017696517299980427, P132=0.00023038005399434182, P133=0.00023184596937088448, P134=0.0003516243804948811, P135=0.0004212934643775088, P136=0.00031877493254037697, P137=0.0005984044216055112, P138=0.0005095581342403089, P139=0.0007022554960406258, P140=0.00029872167435307897, P141=0.0002800038232150533, P142=0.00022391701842407123, P143=0.00021513136956102035, P144=0.000272377786074211, P145=0.0004032515847759861, P146=0.0003085465813941991, P147=0.00019697650061181, P148=0.0001637947685843636, P149=0.00011938548782256386, P150=7.041641559870649e-05, P151=0.000143149458945254, P152=0.0003397364939101037, P153=0.0008200191190606282, P154=0.0005022643621584459, P155=0.0005076566760056176, P156=0.0006934438718693674, P157=0.0003588737719992405, P158=0.00038114667242164995, P159=0.00027779769034585474, P160=0.00011972806519206418, P161=0.00010414813098152674, P162=0.00015079610817885977, P163=0.00010602572307752619, P164=9.864355690855276e-05, P165=9.917400472977809e-05, P166=6.503148346500895e-05, P167=4.694044842621259e-05, P168=0.00014074249213504164, P169=9.625863933978436e-05, P170=8.357398442997232e-05, P171=6.646514714639447e-05, P172=5.5553736667487145e-05, P173=6.589126650746349e-05, P174=8.377488661510246e-05, P175=9.811950529410167e-05, P176=0.0001220885529058039, P177=9.009954352540527e-05, P178=5.210594890126734e-05, P179=5.834530681980868e-06),Row( P0=-474.4116505607483, P1=40.763275982834415, P2=-42.27635460214646, P3=-4.736816873326943, P4=-25.407981976576213, P5=-20.760644078028218, P6=-9.026025598586225, P7=-4.980157752350964, P8=-3.394062819358348, P9=-15.954211092508716, P10=-5.180548110281319, P11=0.46311870117638276, P12=-7.108173215539655, P13=7.8907425188344975, P14=6.741417393995479, P15=5.185906387965401, P16=10.520324280079704, P17=0.736556312204927, P18=7.311011507691365, P19=4.8764223721974975, P20=-0.5747014450773971, P21=-2.5630701093262696, P22=-2.7437484900001836, P23=0.4996407911572293, P24=0.8264120763267591, P25=1.014465293936089, P26=-7.087565291882736, P27=-3.303339554508145, P28=-4.087833812231456, P29=-0.6430062564061315, P30=-0.8919359525684679, P31=-1.7990921598876868, P32=-2.742227811483339, P33=2.245505785962399, P34=-1.7710465844512329, P35=-0.6099255682958478, P36=-0.46359958957198655, P37=-0.8169854437938365, P38=1.8221740990126978, P39=-1.3087348440895177, P40=0.6979923179671603, P41=0.6190962087943792, P42=0.5377786695519476, P43=0.5381524237434887, P44=0.5217716476132369, P45=0.526945327712972, P46=0.5427940042869754, P47=0.5507709666633087, P48=0.5320128057553134, P49=0.5352548293025662, P50=0.5703060749847533, P51=0.6151018259013461, P52=5.945717494591513e-06, P53=1.700032744963927e-05, P54=5.21819094926809e-05, P55=7.485489234019597e-05, P56=0.00012115226780342602, P57=0.00015510741599716855, P58=9.552505915819166e-05, P59=8.912772882088715e-05, P60=0.00012299135880869638, P61=0.00031814092115225903, P62=0.003699641089707796, P63=0.015817341171344437, P64=0.05591369783357155, P65=0.35507599370524473, P66=0.7995993653132154, P67=1.0879350716648164, P68=0.7240387222115544, P69=0.46182036242065155, P70=0.5798543491382633, P71=0.953225606698356, P72=1.2432808563818893, P73=1.420563565410522, P74=1.0691259369802293, P75=0.515926422679111, P76=0.2484771407698773, P77=0.04558231513102038, P78=0.0270166101337995, P79=0.03366398360949991, P80=0.10673944372491874, P81=0.3838074960570273, P82=0.436094750210402, P83=0.4134074588780543, P84=0.7023498939492441, P85=0.810197830061745, P86=0.28466507841556593, P87=0.07791605513495337, P88=0.2518843276818492, P89=0.4080861519825487, P90=0.15680927902364206, P91=0.3111821346086807, P92=1.3712697646100533, P93=0.6640662868653773, P94=0.693766373493269, P95=0.4280255353573637, P96=0.2990551689742421, P97=0.2981242945163939, P98=0.2491387443334318, P99=0.08465912515868573, P100=0.26985807514730903, P101=0.3578511875449701, P102=0.06814950975517285, P103=0.025486921124061677, P104=0.3621393699963055, P105=0.7381101673895613, P106=0.7606572660833532, P107=0.21699347336126845, P108=0.10015570448252975, P109=0.12364930586062364, P110=0.048556165445040385, P111=0.025852845754088465, P112=0.10103682173176627, P113=0.12432180498860655, P114=0.06845443776387941, P115=0.08515240992077741, P116=0.1284870356418503, P117=0.052865361434580385, P118=0.009463769672461004, P119=0.009340356260113243, P120=0.007871838690046365, P121=0.031058579097604785, P122=0.10044294566027338, P123=0.0917837313403735, P124=0.01576852173642901, P125=0.007452492340142543, P126=0.0356499715451023, P127=0.11669279102425398, P128=0.07633019268271478, P129=0.027941757375115776, P130=0.01695008313397783, P131=0.06029856008011124, P132=0.045649914843543776, P133=0.011223745083996574, P134=0.011560932680788552, P135=0.015854474951439896, P136=0.0104492005689441, P137=0.004726013772142992, P138=0.003908850657347551, P139=0.0028524710838318114, P140=0.0017700489998818255, P141=0.001589070202598304, P142=0.0017427131758341267, P143=0.003366726752285898, P144=0.0028375649953882092, P145=0.0050403680655070554, P146=0.007910764007669107, P147=0.011192381338653983, P148=0.004453746020386748, P149=0.003354522328086571, P150=0.0023659547085540054, P151=0.0034055749616921445, P152=0.00401435928920305, P153=0.004602005724667752, P154=0.0030261139104820704, P155=0.0027602670764188654, P156=0.0013732795741140406, P157=0.0007533933615980646, P158=0.0007391490276818101, P159=0.0012963879000864812, P160=0.0030950053981362493, P161=0.0021673390733176466, P162=0.0023387675391621653, P163=0.0017855824168816294, P164=0.0013843326166223962, P165=0.0008238225708328718, P166=0.0009866709748451799, P167=0.0012897513551747798, P168=0.0014348857646266553, P169=0.0014902555544049922, P170=0.0015966689475865952, P171=0.0007864119720496368, P172=0.0004110940772803865, P173=0.00025667028282514733, P174=0.00033092640565265366, P175=0.00038216778490642495, P176=0.000296249680354788, P177=0.0003446297650076251, P178=0.00023382514662824414, P179=3.250546831683054e-05),Row( P0=-390.0428048341515, P1=12.474920434523224, P2=-41.6971583794412, P3=-6.887738753534106, P4=-25.258556061362945, P5=-26.424298035750496, P6=-6.5657624056304265, P7=-5.133370758038719, P8=8.6916976514365, P9=7.44347475973127, P10=5.121180623719483, P11=17.343637698101293, P12=-3.020612685208873, P13=0.3214112813814154, P14=0.6581977532983814, P15=-3.3930720868492177, P16=-0.20312405664025177, P17=-2.5412504276526167, P18=-2.075506646160648, P19=2.8454449982464696, P20=-5.175791131176481, P21=4.086919284994375, P22=0.4441152873849775, P23=-2.0149596288974507, P24=0.7377296596342624, P25=0.9038776314889267, P26=-2.1423774149558237, P27=0.5357158548541058, P28=0.5586744971817199, P29=1.1406912352857286, P30=0.15221942067045038, P31=2.0636798509508485, P32=-1.716218195611632, P33=1.3044632653832193, P34=-1.5493087259880547, P35=0.9822399458914669, P36=-1.084326156214393, P37=-0.8885408980194596, P38=-2.4450548868522333, P39=0.9467537100077826, P40=0.6122674125878635, P41=0.5521735436752094, P42=0.5744200438960054, P43=0.5722308053218303, P44=0.5630843935205416, P45=0.5624755676664873, P46=0.6130036665099577, P47=0.6223390594934838, P48=0.656363900697833, P49=0.6891216016194166, P50=0.6987142412527904, P51=0.676331875275878, P52=0.002582363965609562, P53=0.0023020868155592847, P54=0.002587191473055556, P55=0.0037062907628822384, P56=0.0018994859779443481, P57=0.0007744482598601208, P58=0.0008761192488780563, P59=0.0008633324873793793, P60=0.0007481294931640594, P61=0.0008492210465418784, P62=0.0008380775640768863, P63=0.0006246457324977103, P64=0.0004189882605750266, P65=0.0012403371491138114, P66=0.002792017931568167, P67=0.0030014281611724664, P68=0.008972600065752913, P69=0.08491134532204009, P70=0.1967867967545353, P71=0.2380822357136325, P72=0.39071906427040376, P73=0.5887284781924739, P74=1.8202555395080937, P75=2.9369636909541295, P76=6.299635842405748, P77=14.620859642254565, P78=15.109185952753451, P79=28.00878897274993, P80=21.52009939903479, P81=7.468121819934984, P82=10.314430878181883, P83=13.0957395782491, P84=8.900806302266197, P85=4.232268695427869, P86=1.7404455963301932, P87=1.8564539334105625, P88=4.79215376066666, P89=2.84048445437723, P90=1.003239152138669, P91=0.6690590966555738, P92=0.466657739999286, P93=0.5845134617056256, P94=1.5669404945457026, P95=1.452002531532901, P96=1.089926005825884, P97=1.2562607482287076, P98=1.0781670603690159, P99=0.5134418126461651, P100=0.7088402069885742, P101=0.8093116538008744, P102=0.6656010398871522, P103=0.5948631095561155, P104=0.5763726805588545, P105=0.6493786782313257, P106=0.7987274324750544, P107=1.0950108236715892, P108=0.880600805437568, P109=0.8440282087807743, P110=2.101023678065042, P111=0.6821002449039865, P112=0.3864616445579614, P113=0.39720168275474, P114=0.5622033441138068, P115=2.5124174870519482, P116=1.0334708262271781, P117=1.003125759173276, P118=0.4457225027774223, P119=0.25357333225662804, P120=0.245426747928738, P121=0.5685325568440167, P122=0.9758362436225004, P123=0.882573986627452, P124=1.0411496103066857, P125=3.196069512445537, P126=1.9585544335552898, P127=0.10842547949330489, P128=0.15622751163956858, P129=0.26134125655699136, P130=0.08743269420689027, P131=0.10201687281480462, P132=0.08843931635813228, P133=0.07641093178842387, P134=0.15749486033648807, P135=0.20075385715980376, P136=0.14876752409639946, P137=0.1407173586894986, P138=0.1206901046572674, P139=0.0734632429876048, P140=0.03414284935781338, P141=0.03527039699135082, P142=0.10261384474896289, P143=0.25916881845757644, P144=0.2985797424195145, P145=0.2968599701569331, P146=0.3969354222362024, P147=0.2643910459177098, P148=0.07189697402089082, P149=0.045573224465101696, P150=0.027349115840018492, P151=0.010955833385397505, P152=0.0034193589703717508, P153=0.0024261900873553243, P154=0.001717690159755604, P155=0.00110667281551415, P156=0.0009472217598734262, P157=0.0006351136111454923, P158=0.0009739010655114728, P159=0.0028892692471901966, P160=0.0038759990369456295, P161=0.005456290149616785, P162=0.0045773344290734605, P163=0.0028755252680366656, P164=0.0016983892331709936, P165=0.0026616162054675305, P166=0.004499962191356305, P167=0.003874084009710522, P168=0.0037815271575228427, P169=0.002672548609194261, P170=0.0017444533898948336, P171=0.0018872482043234945, P172=0.0020199450318572273, P173=0.0013345924768263127, P174=0.0010222745039731233, P175=0.0010552928901992828, P176=0.0013613318486757244, P177=0.0014191581351758406, P178=0.0006209067845793921, P179=7.575774637507355e-05),Row( P0=-575.4089659026246, P1=50.345224415565674, P2=-19.675372221447507, P3=-9.455727689296648, P4=-13.066416835503437, P5=-15.776052167731306, P6=-5.590530649044593, P7=2.727836077866771, P8=1.9954669465063781, P9=-11.585218376222416, P10=-8.694596691903907, P11=-1.8704296069624577, P12=-1.5837124305693544, P13=0.04279745793271262, P14=8.118718905824062, P15=15.198953853168462, P16=14.158600278615443, P17=14.120012297635457, P18=9.698153095781775, P19=5.4108874023087825, P20=-1.5185504943231023, P21=-6.146717776889227, P22=-2.32184064748117, P23=-4.0731360678010935, P24=-0.27101641781127994, P25=-1.6676901110149192, P26=-10.825263468180305, P27=-4.577548336542469, P28=-5.773302714453029, P29=1.2166025046949709, P30=1.6035674661359836, P31=1.7565201532861847, P32=-3.7988580633763305, P33=0.6328339987241502, P34=1.1019207381754348, P35=3.3306950764575194, P36=-1.1384029894783987, P37=-2.9490397323255464, P38=-1.5855279660597492, P39=2.398950628724802, P40=0.6962489022720402, P41=0.5597854113949557, P42=0.4731026090215691, P43=0.46581483221888415, P44=0.44244548437194436, P45=0.4620032932571227, P46=0.4989146382069802, P47=0.5450392425165975, P48=0.5107651824732786, P49=0.5054213596052444, P50=0.6082578945044438, P51=0.6643046432399624, P52=7.891030701553861e-06, P53=1.8048252169894335e-05, P54=3.271046248548927e-05, P55=3.344234773869145e-05, P56=4.517356701857823e-05, P57=6.443513793829076e-05, P58=3.02341940789721e-05, P59=1.810204849163337e-05, P60=2.4555582383240463e-05, P61=3.6262241664783e-05, P62=0.00010036850561037377, P63=0.0003949664200907173, P64=0.005404785144024624, P65=0.09083102721546912, P66=0.49874212460420236, P67=1.0458788755061355, P68=0.5240887602070706, P69=0.17499888567784855, P70=0.31241133252656056, P71=0.1208321569989156, P72=0.0034122165224856698, P73=0.001839035991046422, P74=0.0008767544466042779, P75=0.0007730263893580949, P76=0.0004991451558029836, P77=0.0006506038204324279, P78=0.0010110678721095855, P79=0.00367269758752362, P80=0.01610914583694953, P81=0.026435444361977027, P82=0.09671679342024231, P83=0.38123873731665153, P84=0.5513053660598813, P85=0.8016656499925998, P86=0.2908876468426236, P87=0.03762748347687399, P88=0.1537840510960426, P89=0.08946407928975252, P90=0.00362511722678391, P91=0.0009600230167477657, P92=0.005026226074831993, P93=0.011520744488769398, P94=0.038804609795358457, P95=0.30667758556472474, P96=0.1085977255424233, P97=0.013190865399565098, P98=0.009242712781818842, P99=0.0002551797690565931, P100=0.00047303162609302265, P101=0.0016217178673006358, P102=0.0012220105762712223, P103=0.025815604383947408, P104=0.12308864854351895, P105=0.012740911427007902, P106=0.00285667942682794, P107=0.00046405960648429964, P108=0.0003326989669512531, P109=0.0004475771743127438, P110=0.005653717668497611, P111=0.015925306736100747, P112=0.0035733388801542265, P113=0.002466633853598988, P114=0.0010776212129704607, P115=0.002613258083041231, P116=0.00863964425086401, P117=0.0060719782100256406, P118=0.0033724171185522123, P119=0.003375342264295413, P120=0.0008419500484336458, P121=0.013936929903257482, P122=0.014506234993429802, P123=0.0017319421095021324, P124=0.0006405547565066586, P125=0.0005935855052799166, P126=0.0012049302413582176, P127=0.0011348318073218855, P128=0.00020617608337136816, P129=0.0003429445887631724, P130=0.00022997293471509824, P131=0.00015681336720243223, P132=0.00041786740213672256, P133=0.00017124006968849835, P134=0.00020589811731866142, P135=0.0004062803357402147, P136=0.00014856009131103645, P137=0.000184311053574505, P138=0.00019620671813473742, P139=8.951055903268877e-05, P140=0.00012576944242343453, P141=0.00010976180585508066, P142=9.492150273793536e-05, P143=0.00013667096184183944, P144=0.00016439583862588842, P145=0.00013008270655615997, P146=0.00014035032824237972, P147=0.00012116594438404889, P148=0.00021988814429944786, P149=9.119861712002749e-05, P150=3.197841869041348e-05, P151=5.305029563561512e-05, P152=0.00018226851209964193, P153=0.0003682131262294815, P154=0.0003525748064741751, P155=0.00028542273946610447, P156=0.00041647557711448416, P157=0.0003977835988451448, P158=0.00016310409562935777, P159=9.051243288987252e-05, P160=6.158756415438458e-05, P161=3.438286415788788e-05, P162=4.411582123376763e-05, P163=0.00012746197081226767, P164=0.00016091622435742718, P165=0.00017200800399550477, P166=0.00019427328443053884, P167=4.706235120463744e-05, P168=2.3563962040392534e-05, P169=7.681870754462407e-05, P170=8.287595576646289e-05, P171=8.969188451614405e-05, P172=6.572223875591173e-05, P173=2.4911394879060947e-05, P174=4.458634199385146e-05, P175=6.0754937406245226e-05, P176=0.00011016127213283128, P177=9.63194807060271e-05, P178=3.609069016824358e-05, P179=5.864815072524785e-06),Row( P0=-594.9319210347315, P1=54.356611718738094, P2=-13.943637060405402, P3=1.876777466509322, P4=-12.50051010082827, P5=-10.465150751014692, P6=-3.296866243154842, P7=-0.4684666181408626, P8=0.7873496863287855, P9=-6.832245850884835, P10=-0.6829179258890637, P11=-2.3027152125711123, P12=-6.584178806556332, P13=-3.7048942252376107, P14=-0.45073183087764224, P15=-0.8197360783297092, P16=1.620077052443935, P17=4.968637174990315, P18=3.7138827360909445, P19=10.738227536155142, P20=8.974446609383643, P21=8.806580441867844, P22=7.162565584520676, P23=-1.2609789001922072, P24=-2.1325543843692603, P25=-4.654988866814949, P26=-5.065769443247275, P27=-0.24905396364923668, P28=-3.9153616115552645, P29=-0.275170272926832, P30=-6.395571679809832, P31=-2.1047826827251006, P32=-6.015134487981006, P33=1.1369920312506123, P34=-4.638522003405011, P35=-3.609216832452291, P36=-3.523893003983537, P37=-1.801304998504479, P38=-0.026949474503885414, P39=0.3855164264891928, P40=0.5625902584549033, P41=0.586472871405458, P42=0.594175737729667, P43=0.5798845478331751, P44=0.5647318397461216, P45=0.5671556756761789, P46=0.6566546247340883, P47=0.717960666810118, P48=0.7562007259366424, P49=0.7090572706240331, P50=0.6567689036829475, P51=0.58090604254008, P52=5.991512382544345e-06, P53=2.3457777880457895e-05, P54=7.47352700510372e-05, P55=0.00013755279244815092, P56=0.00017341480133636352, P57=0.00013807302220340866, P58=6.515228543080542e-05, P59=5.190590542147452e-05, P60=0.00010930383552300946, P61=0.00048506391800482953, P62=0.014234114968727788, P63=0.10089879000525699, P64=0.2553854112725929, P65=0.3110776826261334, P66=0.1768599556712671, P67=0.0550881996938587, P68=0.0752169775130784, P69=0.15179718949550655, P70=0.057675265545862935, P71=0.0059282506009527966, P72=0.0015129620883853573, P73=0.0015809580920935669, P74=0.010530253814562945, P75=0.029647184640372297, P76=0.06966130124358712, P77=0.2853409669955733, P78=0.21720407229904637, P79=0.11901031866451352, P80=0.07699882776085674, P81=0.0272510316113767, P82=0.010344696271663697, P83=0.006696575778887302, P84=0.012690911993857924, P85=0.05027872698397269, P86=0.1308092783390803, P87=0.10654744166156559, P88=0.05108423293007519, P89=0.1386166911816173, P90=0.09543695760428661, P91=0.036190716085884374, P92=0.046544237169742785, P93=0.012923193965851221, P94=0.007093048657395407, P95=0.007005571077818105, P96=0.007812697712886568, P97=0.028473634339723126, P98=0.026382910436332285, P99=0.006649636733625628, P100=0.050719801489653764, P101=0.0475566542536495, P102=0.009569802031916744, P103=0.005268084143645092, P104=0.006215616980572045, P105=0.005017508344942642, P106=0.002297160504311081, P107=0.022316509511067267, P108=0.011840931291969066, P109=0.00206172376023034, P110=0.0027033898221608403, P111=0.0034407370390022373, P112=0.007052351155638763, P113=0.027563406441607555, P114=0.0038479687366072277, P115=0.0026787568930415743, P116=0.0007287117911833794, P117=0.0014034937927965819, P118=0.0015259522501493573, P119=0.0010160861797547482, P120=0.0011319810468256467, P121=0.0015131196299471533, P122=0.0016654538176796674, P123=0.0011637439501277934, P124=0.0012547322793348863, P125=0.002484193811604021, P126=0.0012716098106419535, P127=0.0010894022133256373, P128=0.0020821064043295865, P129=0.004680792034462368, P130=0.001105040425349647, P131=0.000781207955376152, P132=0.0008937572682246562, P133=0.00036397238740787597, P134=0.00031161557813786154, P135=0.00024851907934116287, P136=0.0006213683005559021, P137=0.0004994403846088833, P138=0.00022441205863619175, P139=0.00028049436026070625, P140=0.00011167298606824329, P141=0.0001730026485050005, P142=0.0001618738656087637, P143=0.00025005885431280675, P144=0.00041921759844028625, P145=0.0005757894650701768, P146=0.0007168743386690692, P147=0.0003861570576616179, P148=0.0007618344822899385, P149=0.0004023288130259757, P150=0.00017119619544538357, P151=0.000256287306289141, P152=0.0001774837785970917, P153=0.00010769252442864679, P154=0.0001401972766237524, P155=0.00011062297009478316, P156=0.00015174638364161492, P157=0.00018516883660034324, P158=0.0001916744676160039, P159=0.00023791898350097545, P160=0.00015101864944752963, P161=0.00012559602453738583, P162=6.376083783987589e-05, P163=2.958674581953178e-05, P164=2.8739695376569498e-05, P165=4.023742577644025e-05, P166=3.509814446507728e-05, P167=4.86834823473645e-05, P168=4.354606439557663e-05, P169=3.354491957391517e-05, P170=2.4036988154061447e-05, P171=1.193019579448603e-05, P172=3.488738898855681e-05, P173=5.910255899356747e-05, P174=5.350411300421666e-05, P175=6.413835589289395e-05, P176=6.776807874052341e-05, P177=4.0495432624374236e-05, P178=2.7101928734769618e-05, P179=3.2407974605944186e-06),Row( P0=-561.0488437674784, P1=40.80914131673143, P2=-15.770027901499532, P3=-4.038895786399814, P4=-22.4762887048451, P5=-14.772554248771673, P6=-6.547292329190737, P7=-6.263354428298664, P8=5.20990603119825, P9=-8.991619030726484, P10=-3.0349490828576466, P11=-1.2130777475298138, P12=-5.562997992767945, P13=-4.932732333448464, P14=3.032337918877388, P15=6.5506726573409555, P16=14.925456505085968, P17=19.710126097506183, P18=11.052818349788906, P19=10.240226808625081, P20=0.8583719520410719, P21=-3.6964178087894877, P22=0.6523091639272784, P23=-4.193136104417778, P24=-0.09528265455691134, P25=-2.8406271779536407, P26=-8.674937952697302, P27=-4.8645120353277305, P28=-5.757356421830143, P29=1.0629591798978653, P30=-3.354319202000353, P31=-0.07503182834755284, P32=-3.081437029967902, P33=2.6133787849945236, P34=-0.5448070770787838, P35=-1.7697237158812096, P36=-3.2428270309742984, P37=0.39048731210812737, P38=1.0207513746481667, P39=0.355846193743849, P40=0.7166740571893482, P41=0.6031520759440869, P42=0.47055436758986063, P43=0.41348984576653963, P44=0.48301088496769595, P45=0.5059932365201767, P46=0.4964161371146518, P47=0.5355711888055064, P48=0.5816190960205136, P49=0.6704165726597834, P50=0.739476434414075, P51=0.6865708157664596, P52=6.0495862994763375e-06, P53=1.8544169083174854e-05, P54=4.0700826671983336e-05, P55=4.265159511733057e-05, P56=2.9147278197705067e-05, P57=3.1660103441801415e-05, P58=2.115249965681374e-05, P59=1.12862574624759e-05, P60=1.4946505425472773e-05, P61=2.3569272262076834e-05, P62=0.00019924376249715473, P63=0.0017871308767687596, P64=0.030228709313373396, P65=0.25175908184218954, P66=0.742774355696389, P67=1.3057647780489121, P68=0.6256567158340766, P69=0.11249473518374817, P70=0.023941554855712723, P71=0.0037795846530258703, P72=0.0011246563708936445, P73=0.000745474995380529, P74=0.00042755624381298754, P75=0.00036188892547834434, P76=0.00030744533707644086, P77=0.0012382924463961345, P78=0.020198644985923123, P79=0.0838701258658656, P80=0.10515571638595751, P81=0.13417046974111477, P82=0.24014351126493383, P83=0.4359886373740594, P84=0.5210280731342284, P85=0.26651470450163495, P86=0.05788643370599986, P87=0.01055314428752867, P88=0.004340554093807554, P89=0.002514331739786746, P90=0.004814348837428453, P91=0.06816098728993371, P92=0.06338220951055984, P93=0.009483102840134726, P94=0.006053613086968693, P95=0.01595924326800163, P96=0.12361319922797075, P97=0.09899414347668534, P98=0.08092197201266753, P99=0.007309734087969277, P100=0.005826750181438861, P101=0.0028083594829458867, P102=0.0009534256947829939, P103=0.0003126375625652621, P104=0.0028629882186855435, P105=0.02761010413020846, P106=0.030918330074032, P107=0.013929291016430595, P108=0.0041397820606428395, P109=0.0008705337134516538, P110=0.00015482662975993506, P111=0.0019900725863342826, P112=0.010932822847962754, P113=0.010845598550358448, P114=0.00784095533049291, P115=0.0010542332992859835, P116=0.00015297818011810916, P117=0.0027052834246567925, P118=0.004368100381456271, P119=0.0015176192377790582, P120=0.001265603652601034, P121=0.0017257877600007742, P122=0.009734850098809948, P123=0.0065470144411772435, P124=0.0008225785063248609, P125=0.0015058056854922962, P126=0.0032958660800506807, P127=0.0022943538958298034, P128=0.0005905071502087187, P129=0.0006504343405495036, P130=0.0004193457307202873, P131=0.0001677673969162315, P132=0.00018948046907506144, P133=0.0008773529393244927, P134=0.0005149942588926362, P135=0.00036256888600019803, P136=0.0003529764680337297, P137=0.00030328918399290207, P138=0.00015007929424234074, P139=0.0001430923496180637, P140=5.1035645447839004e-05, P141=6.182326917390357e-05, P142=8.314077262080194e-05, P143=0.00017104547532071582, P144=0.0002631684540835148, P145=0.0004536725217069812, P146=0.0006333225786937838, P147=0.0007092308131455234, P148=0.0007950936027026484, P149=0.0006332835929638757, P150=0.0003829830112932668, P151=0.00011333196990732559, P152=0.00024381705815375342, P153=0.000428801400981296, P154=0.0009839665521131275, P155=0.0008542727441921876, P156=0.000694293344736229, P157=0.0007409972442983523, P158=0.001062691565036824, P159=0.0008593462505240394, P160=0.0011357271984001026, P161=0.0006350397860074841, P162=0.00026565173790655844, P163=0.00017388244081805885, P164=0.0003794606252389459, P165=0.0005801669372471051, P166=0.0007633851169134858, P167=0.00047468603463630667, P168=0.00025618705249907475, P169=0.00013430170340251345, P170=0.0001339100176496894, P171=0.00015155191757919086, P172=0.00015369294576322658, P173=0.00012576362715938335, P174=0.00010565608002355364, P175=0.00010992367007693091, P176=0.00013053269273538792, P177=0.00020583335209263897, P178=0.0002235698825470777, P179=4.525449606059318e-05),Row( P0=-517.3447783824236, P1=41.87396201569543, P2=-36.39475553623853, P3=-7.121385937093769, P4=-19.459980496341736, P5=-18.53536939892703, P6=-9.679589304080425, P7=-4.1698782703661745, P8=-1.00681275076092, P9=-17.97647984228559, P10=-2.3452767122187335, P11=1.106101886794497, P12=-4.274128548240847, P13=13.221330964533271, P14=9.756911874107267, P15=14.0016184912945, P16=12.45423275638558, P17=-0.8152692152923839, P18=4.573588591159833, P19=-0.22288662070688525, P20=-1.4987693631223826, P21=-2.46841026369112, P22=-1.5296097583918307, P23=-3.4395685444753386, P24=-1.4818982493274138, P25=-1.7326277759430349, P26=-5.950432652511361, P27=-0.6060601282755536, P28=2.6240794026434506, P29=-1.638774321900689, P30=-0.2995606454113104, P31=-1.6991423463472277, P32=0.04373748848372374, P33=3.622100349848453, P34=-3.078131413678887, P35=-0.5270698875267337, P36=-2.602880415553273, P37=3.6662447636894857, P38=1.125109443453896, P39=-0.675730779034574, P40=0.7066339203458116, P41=0.6641825820214666, P42=0.6051778074154932, P43=0.5360637939437849, P44=0.5187811405091658, P45=0.49824497719618277, P46=0.48906223336270216, P47=0.4842271649467127, P48=0.5144082200130772, P49=0.5182467511650869, P50=0.5333629686772033, P51=0.6208299071169824, P52=8.149097609226351e-06, P53=1.838368376786863e-05, P54=3.228615647501753e-05, P55=4.635633794766312e-05, P56=6.214781409874518e-05, P57=0.00010087302208517918, P58=7.142795172892689e-05, P59=6.041328799066016e-05, P60=5.8012139640552925e-05, P61=6.21824120013019e-05, P62=0.0001162822271157105, P63=0.0004386960766081564, P64=0.0063566286482892274, P65=0.07844053045993739, P66=0.32481670263251544, P67=0.5392122225874916, P68=0.6982322834841969, P69=1.4843017466837787, P70=1.437353264272358, P71=0.6079621406705386, P72=0.715369637865074, P73=0.8647874461240671, P74=1.0908175961562587, P75=0.3785004016413089, P76=0.029261083144717193, P77=0.006268539726127416, P78=0.006949714103105001, P79=0.005912010332712347, P80=0.007335353929258037, P81=0.012424900388424925, P82=0.025488190790508975, P83=0.06266564051582911, P84=0.1882245453413887, P85=0.40599522080008943, P86=0.4886597478464896, P87=0.5965647230752048, P88=0.8351949351964785, P89=0.7343960517203955, P90=0.43134885900982173, P91=0.6090488274732899, P92=0.49464938770705863, P93=0.38180029608664723, P94=0.6538528135634573, P95=0.443483754256359, P96=0.02596173813572973, P97=0.15534076756110513, P98=0.2797836539475637, P99=0.088795483877441, P100=0.05389906346979017, P101=0.051310562549482665, P102=0.019446931021033953, P103=0.029350015675782017, P104=0.12292019762672399, P105=0.284021762564375, P106=0.4737892582369681, P107=0.3177692938544273, P108=0.06252725888729398, P109=0.006333181357022816, P110=0.005026148295592414, P111=0.004724586210348214, P112=0.029016602011068857, P113=0.036288305169159735, P114=0.03182116803995613, P115=0.04118773803542568, P116=0.01048487654686242, P117=0.006893643444228945, P118=0.019084364751594755, P119=0.03252388880922053, P120=0.022619579201707436, P121=0.014444737534663899, P122=0.015697011279754133, P123=0.019803434571075036, P124=0.0282100491690193, P125=0.023827948259410946, P126=0.008049379335252238, P127=0.01355184949165554, P128=0.026331206290867302, P129=0.02155283522085259, P130=0.00739066251403443, P131=0.014532898166609616, P132=0.014573955781524768, P133=0.0062939033176720805, P134=0.008788588890986212, P135=0.0129446022858902, P136=0.01031071237457319, P137=0.009245834781050141, P138=0.0016539838336220808, P139=0.0005463088986794435, P140=0.0007362095086121187, P141=0.000220130005354453, P142=0.0002033168147317628, P143=0.0005181504977832429, P144=0.0008935322487149662, P145=0.0015006115404215452, P146=0.0028143682024580606, P147=0.0038184177962812295, P148=0.0024198685576612288, P149=0.00175488509243706, P150=0.0010368243007354656, P151=0.0017741554307660551, P152=0.002644034694787497, P153=0.0021062037901192657, P154=0.0011506061201528657, P155=0.0007111063600932693, P156=0.0005867567893805304, P157=0.0002637678924317041, P158=0.00011750064633960634, P159=0.00018282545702763542, P160=0.0005688425291001194, P161=0.001076338279659585, P162=0.001253269771048765, P163=0.0010300587219671915, P164=0.0004203352470200897, P165=0.00013330441138949067, P166=0.0003176230315687955, P167=0.0007114147911787778, P168=0.0012251685695468739, P169=0.0012675118714487827, P170=0.0007610406850261151, P171=0.00032159718105609315, P172=0.00016028937930055967, P173=0.00013914838912146383, P174=0.00016361086858042082, P175=0.00016020759197970754, P176=0.00022288217426228872, P177=0.00037520097703098, P178=0.00027008196167138616, P179=2.5694875751409414e-05),Row( P0=-557.1436294274295, P1=40.938420142980235, P2=-23.308577603527645, P3=-7.032285631682889, P4=-21.625603648880592, P5=-16.020267814683326, P6=-6.485025823702193, P7=-4.675734366474924, P8=3.0539829246142065, P9=-6.11937323031302, P10=0.8492214087260658, P11=5.960093810128999, P12=1.8129086032530481, P13=9.310515780066597, P14=11.038759086350199, P15=5.147233670014984, P16=8.345139255597287, P17=-2.5628506297302356, P18=3.5524747717789538, P19=5.5482709034270075, P20=-1.0070584621029397, P21=0.7980317525962048, P22=-6.887629513237974, P23=-5.495141579288232, P24=-2.1986684502232174, P25=1.8360205499579338, P26=-0.9678799514515752, P27=-3.106338653653862, P28=-3.1796371620877144, P29=-2.608693204881449, P30=-1.624260592269987, P31=0.4375602469056021, P32=-3.38402171070664, P33=1.6329818181507445, P34=-0.6883890372292961, P35=0.46773295152548616, P36=-0.7110841846435121, P37=0.6197029635758611, P38=-1.096795423612404, P39=1.4022204644161282, P40=0.6260545657443168, P41=0.5469634299314773, P42=0.5236360876221089, P43=0.5391507815566835, P44=0.5768717766714041, P45=0.578667851347355, P46=0.5236564071618809, P47=0.4967916394681525, P48=0.5051764480027657, P49=0.5515105306743355, P50=0.6084045005012507, P51=0.6591237916197834, P52=7.307989653497682e-06, P53=1.7756111117966757e-05, P54=4.31726273739801e-05, P55=4.6266651979310445e-05, P56=6.945188041377481e-05, P57=8.135312447456487e-05, P58=3.390943961215386e-05, P59=2.572665205061897e-05, P60=3.261957621107404e-05, P61=2.858412548491386e-05, P62=3.677334240422676e-05, P63=7.901644188597425e-05, P64=0.0011033650473305698, P65=0.04462040823017841, P66=0.1213128542326924, P67=0.16102207686073258, P68=0.1431268596937593, P69=0.12980658821034446, P70=0.17586241118957888, P71=0.28570494574075633, P72=0.3903417485920795, P73=0.362724461156297, P74=1.0656452141252477, P75=0.5193356325033619, P76=0.08852447198687725, P77=0.04529785257482675, P78=0.045951583991246485, P79=0.06533965934619371, P80=0.09958339180593537, P81=0.10549619187399903, P82=0.10201037249368049, P83=0.1000939543825417, P84=0.16231269803719928, P85=0.21401945729153737, P86=0.12948234143234522, P87=0.1188606355047087, P88=0.21581611719984173, P89=0.36339260302028425, P90=0.6179802767006513, P91=0.7979912159153739, P92=0.47194078391545113, P93=0.13880489277774755, P94=0.10126594630763591, P95=0.0334887395414756, P96=0.004732030950428793, P97=0.008242193971759845, P98=0.014446778043973811, P99=0.020877284326470128, P100=0.05164685239548693, P101=0.08181793444605906, P102=0.08326412970443213, P103=0.22327111234547234, P104=0.19284633759289058, P105=0.026171376720172564, P106=0.00872700113616249, P107=0.002875701440264734, P108=0.003947720362386153, P109=0.0037582967058190516, P110=0.002997478523704564, P111=0.006780771624119864, P112=0.01087034726528794, P113=0.01258066740897761, P114=0.008065590471149213, P115=0.005800210745531014, P116=0.004889923223486736, P117=0.003492830300084456, P118=0.0038158264895956228, P119=0.004950139924993746, P120=0.009317906560076144, P121=0.0063149571861722265, P122=0.0024414476865575905, P123=0.0070086537857386505, P124=0.010072937206659645, P125=0.0060102486830282105, P126=0.0036031742822326272, P127=0.0022315249635951697, P128=0.004168678201059451, P129=0.008208027574312942, P130=0.002021521614521581, P131=0.0014352438570560326, P132=0.0015976891388090973, P133=0.003867382961211559, P134=0.005625178010670073, P135=0.005343735051898667, P136=0.0014400870393823867, P137=0.002041690812488896, P138=0.0018395672929973936, P139=0.00028363526541372666, P140=0.00024152976099077236, P141=0.0004746784880179265, P142=0.00027411083767474596, P143=0.0005269516718261441, P144=0.0012531534292296452, P145=0.002687332976138949, P146=0.002929677369972304, P147=0.0015671914547381554, P148=0.0007675600674578964, P149=0.0011825994963161374, P150=0.0021031992476028176, P151=0.003008521963705367, P152=0.0021535631359731002, P153=0.0015232266847202737, P154=0.0010146859174397116, P155=0.0007661732074816218, P156=0.0006355257914099148, P157=0.0003729341525856122, P158=0.0003909684400403306, P159=0.00044144092070779736, P160=0.0008949118079520272, P161=0.001434979388377246, P162=0.002465229282862067, P163=0.00160480132573856, P164=0.001026460369852087, P165=0.0006238388146796112, P166=0.0003410647660724003, P167=0.0003737106950720446, P168=0.0006233658456327917, P169=0.0009568445815954464, P170=0.0007272793627254604, P171=0.0004250861431445645, P172=0.00015714982956396934, P173=0.000223110979566865, P174=0.00018396147383664788, P175=0.00021503511806953243, P176=0.00022581767553524982, P177=0.00016601834647331454, P178=0.00011570470936942586, P179=9.84751329405741e-06),Row( P0=-609.3758123554737, P1=48.312592694462325, P2=-9.784958663289114, P3=-6.890568171526961, P4=-12.907641610411284, P5=-14.26152757955304, P6=-5.516601895451167, P7=2.1493989847716843, P8=2.6374541028487055, P9=-11.179815897382882, P10=-6.222867029111266, P11=3.1716550546814433, P12=-1.6500156294313086, P13=6.324176534639612, P14=14.342361950822362, P15=14.486939244858092, P16=12.911147054417, P17=3.390574879219799, P18=2.429076145904837, P19=2.2276188176443683, P20=0.30159472459859543, P21=2.227505702805716, P22=-0.5539525770088308, P23=-6.77347371403428, P24=-5.296974303679983, P25=-5.291198887689827, P26=-2.8986737224155728, P27=4.522427647770695, P28=-0.9776232431647353, P29=1.101904501090894, P30=-1.0958963467180483, P31=1.5622676600200602, P32=-2.8681227166308614, P33=-1.2984560439867727, P34=-4.426791426810109, P35=-3.0495649761186443, P36=-1.4539653819387524, P37=-1.1832219205423053, P38=0.13171232855396556, P39=-1.7037682324090495, P40=0.6278951771120594, P41=0.652896164806036, P42=0.6676829814780662, P43=0.5908176170684155, P44=0.5374969125839316, P45=0.48705363281835035, P46=0.4890732657851235, P47=0.5645477852622623, P48=0.592626033658565, P49=0.5688909613305893, P50=0.5937232371200727, P51=0.5965234238995882, P52=1.0303548107010499e-05, P53=1.781797913065007e-05, P54=3.5132781862301324e-05, P55=2.4580414253332136e-05, P56=1.9396531545082023e-05, P57=1.7044665003109177e-05, P58=7.037377117775819e-06, P59=4.5803740085469766e-06, P60=6.880820972914688e-06, P61=1.339772381084437e-05, P62=0.00011258769358804809, P63=0.0007925349435867571, P64=0.004628517193561425, P65=0.030974079444524877, P66=0.07503087440436731, P67=0.11160197885261093, P68=0.20514281177926796, P69=0.5939275737501731, P70=0.5756085085459023, P71=0.26490795610851886, P72=0.0810841209557508, P73=0.014448967867350533, P74=0.0037693094991775385, P75=0.002407069637108405, P76=0.0015283528560293476, P77=0.004990224885777284, P78=0.010065047308134158, P79=0.008425814966912352, P80=0.007092103252846071, P81=0.004283180832931984, P82=0.0029839125965126644, P83=0.009601950079633688, P84=0.05549499424011037, P85=0.0758856139715697, P86=0.07165330370091245, P87=0.13847274846654364, P88=0.19661345559008772, P89=0.11128696424553773, P90=0.10753950144086015, P91=0.1527859328965633, P92=0.04326288566996226, P93=0.003010327012622505, P94=0.00033754774074840803, P95=0.000691449772189746, P96=0.015198190399723468, P97=0.01878804505610466, P98=0.003539819489321242, P99=0.001817448485744693, P100=0.002157097666481219, P101=0.014254212848142154, P102=0.050367934397525446, P103=0.022450367108245516, P104=0.0010944036274279662, P105=0.005787798522062888, P106=0.004611831495097603, P107=0.0002890895286559299, P108=0.00034808189269240337, P109=0.0001907539225131153, P110=0.0010301684629606838, P111=0.003877795485055312, P112=0.0033509621816168243, P113=0.001186480409101827, P114=0.00022025357103804701, P115=0.00047491425418735313, P116=0.00041112707453253624, P117=0.0008198015733298313, P118=0.0058193571808261, P119=0.0036053355690899456, P120=0.0012843481220046506, P121=0.0019324452505960963, P122=0.005386469574815828, P123=0.0042045139000907086, P124=0.0008017464210266981, P125=0.0007286672399964489, P126=0.000903425984698785, P127=0.0010213964687360271, P128=0.00016763786382150488, P129=0.00012017191873207964, P130=0.00029773587245762826, P131=0.00018787922535882165, P132=4.666885960949904e-05, P133=0.00012931085615644857, P134=0.0003068389157145823, P135=0.00011114932021934539, P136=0.00010458309488031813, P137=0.0002652439189236399, P138=0.0001762880345540826, P139=0.00013686982332485924, P140=6.32608575583251e-05, P141=5.28040891289993e-05, P142=6.599674792190369e-05, P143=0.00012225493454824342, P144=0.00022933259629390055, P145=0.0005126855073036589, P146=0.0007436343251365494, P147=0.0010371381042550108, P148=0.0006902026298118562, P149=0.0004039244522855299, P150=0.00021573123136097886, P151=0.0001769486058516041, P152=0.00011584084727993588, P153=0.0001728274143463346, P154=0.00018298702760644994, P155=0.00038964395947545184, P156=0.0005405547969276383, P157=0.0005522799125609053, P158=0.0005250043564445379, P159=0.00044721238317182516, P160=0.00037365497358311106, P161=0.00024393242267965696, P162=0.00013845287360456972, P163=0.0001876670692060645, P164=0.0005168271221622575, P165=0.0006507859210601438, P166=0.0005871007094575408, P167=0.0005658771113125503, P168=0.00023479202113330943, P169=8.948318225038203e-05, P170=0.0001902011448484242, P171=0.0004356632715577257, P172=0.0006314069871513775, P173=0.0004690683758637649, P174=0.0002710066733534156, P175=0.0002735893382619179, P176=0.0006345595942233055, P177=0.0005708124868017905, P178=0.0002856204983026828, P179=3.505006063252115e-05),Row( P0=-558.8242030875175, P1=47.25339895622892, P2=-22.15029531928506, P3=-4.4696188649757875, P4=-15.860669339286941, P5=-18.252349288443174, P6=-3.1844967832894704, P7=7.266300083543264, P8=4.510358474507405, P9=-14.110145435240426, P10=-8.377665779466588, P11=1.8264315561375042, P12=-5.387570217187223, P13=5.730137486365805, P14=15.04433985317032, P15=11.592633688920367, P16=10.395389313406707, P17=0.035426376813545025, P18=-0.25778464030355663, P19=2.788861996780354, P20=-2.3934167280698357, P21=0.08440569672290563, P22=0.3111510968938166, P23=-3.612506842932342, P24=-4.676245077219008, P25=-3.6980198116363523, P26=-5.160156009829008, P27=4.405991518878862, P28=0.2557258539932022, P29=0.4721454081364489, P30=-2.961473806971069, P31=1.06366246209217, P32=-2.26923277364253, P33=-2.8891290561568073, P34=-2.217864911880217, P35=-3.537612962256843, P36=-2.2152560956008, P37=0.2659593638523945, P38=0.7105831361623491, P39=-0.9700089537483931, P40=0.6932212835645009, P41=0.6876758384425592, P42=0.7427731735532999, P43=0.6576248102389095, P44=0.5842409570618842, P45=0.5453193588074967, P46=0.4819272447748664, P47=0.5387393934656921, P48=0.63687342721485, P49=0.6183381812804413, P50=0.6011484422717267, P51=0.6325221888392889, P52=2.0949276970662926e-05, P53=3.084590181082049e-05, P54=7.853852565228589e-05, P55=0.0001302284267208282, P56=0.00017159528877327287, P57=0.00021523458593081094, P58=0.00010872839304885187, P59=8.748273080996246e-05, P60=8.97855833773736e-05, P61=8.444662775910677e-05, P62=0.00044726083324760817, P63=0.0021740924563363935, P64=0.009402708218357723, P65=0.06393031022758723, P66=0.14223602605863614, P67=0.19763623814382092, P68=0.5742280352137913, P69=1.3797324222653409, P70=1.0701925304371573, P71=0.4045455330097895, P72=0.29987791249107, P73=0.22911417902631928, P74=0.05682807375640633, P75=0.015559815418301023, P76=0.0076923562069368965, P77=0.006198801761696245, P78=0.009205383450524244, P79=0.01616995423105719, P80=0.021208690295594426, P81=0.01716386523703781, P82=0.021593883429823386, P83=0.03893652072245167, P84=0.08263356371943761, P85=0.1139453793311166, P86=0.1650903344805046, P87=0.47656133356215086, P88=0.451574170235371, P89=0.20474606385884, P90=0.062169430551518835, P91=0.04037648785456075, P92=0.03761172561122429, P93=0.009861650399138772, P94=0.003301509526090507, P95=0.002604900877399136, P96=0.0016690610576439394, P97=0.006686256635886823, P98=0.01735100606291137, P99=0.07082413040627394, P100=0.12267690856541072, P101=0.07362068440066533, P102=0.026616579539280722, P103=0.0022683730529019443, P104=0.0009755543954208357, P105=0.002135434541950656, P106=0.0027855351108128043, P107=0.0042782467775821995, P108=0.03572904318311421, P109=0.023076753465349303, P110=0.007212875719077902, P111=0.0017248204625447854, P112=0.001147437449356863, P113=0.0008075429786715945, P114=0.003314524157667772, P115=0.0058409308760465906, P116=0.0017264599723674245, P117=0.0008019331669530687, P118=0.003004407527132766, P119=0.006188894022632186, P120=0.0027047650147674126, P121=0.0022846671374120945, P122=0.0011921662260344522, P123=0.0014513726678848822, P124=0.0021066837804486066, P125=0.003571076847281361, P126=0.004133927188070944, P127=0.0014667971097174371, P128=0.001309181399058134, P129=0.0009786886121994924, P130=0.0004348184869228418, P131=0.0003585551164486139, P132=0.0005839137503651259, P133=0.0004628985460120173, P134=0.00035036677597386005, P135=0.0004739569801248237, P136=0.0004053679362697604, P137=0.000355498091440512, P138=0.00021845563015020794, P139=0.00021390087017701448, P140=0.00013429235582986286, P141=0.0001730500808386277, P142=0.00041041193842723394, P143=0.0010498004378816726, P144=0.0007850714823732171, P145=0.0012189889458519756, P146=0.002273966141039531, P147=0.0019899476224681583, P148=0.001107518734439832, P149=0.0007355147050704876, P150=0.0007789607097277476, P151=0.0005800163104676594, P152=0.00033510201435457866, P153=0.00022184844295179918, P154=0.00025484686901689114, P155=0.0004880553051612762, P156=0.00048818163007443514, P157=0.00047392965614433266, P158=0.0010512623983774253, P159=0.0012252401366682683, P160=0.0007772026688954223, P161=0.0005111468870258956, P162=0.0004932620571889021, P163=0.000542100713140173, P164=0.0008607380276895986, P165=0.00238476804979726, P166=0.0010432436997229668, P167=0.0005334847830378717, P168=0.0002476580752066088, P169=0.00016211483960903612, P170=0.00015858506132563867, P171=0.0003347261532557465, P172=0.0004844383322839515, P173=0.0003366207846866091, P174=0.00025739976806705896, P175=0.00032201585193930284, P176=0.0001895558803346892, P177=0.00019270080588125948, P178=7.436477811556811e-05, P179=7.678787376665365e-06),Row( P0=-511.62942039079036, P1=28.873275528000445, P2=-25.54427319813614, P3=4.461003219595442, P4=-8.690041888181835, P5=-7.544861620729576, P6=3.673368159307242, P7=-3.0802768011860264, P8=3.3561884670016355, P9=6.784853799665926, P10=8.262039754326088, P11=21.10895617800686, P12=13.957585998926522, P13=15.952099173356169, P14=3.4839731673874734, P15=5.898878925228157, P16=5.324208053252549, P17=5.801537911409955, P18=12.47155695820282, P19=6.867789921480138, P20=2.5600392031121566, P21=5.19154063982938, P22=0.5827381968503675, P23=5.457344764649888, P24=0.6191742145678151, P25=-0.7561377450481525, P26=-1.3507444194200793, P27=1.7188518150488103, P28=-0.7873999919785872, P29=3.1650683783711324, P30=-1.7984989420051423, P31=2.2135495355350767, P32=-0.003709663189005729, P33=2.4241072304845974, P34=0.4916610650224258, P35=-0.048223890366448294, P36=0.794076002642792, P37=1.398530634941026, P38=0.6664606655541621, P39=1.3827035604481985, P40=0.5860514772028796, P41=0.5941420435460503, P42=0.5932453496912931, P43=0.644385258807639, P44=0.6707479195358124, P45=0.6564806991988157, P46=0.6679834743764266, P47=0.6444122647104189, P48=0.626819097854196, P49=0.6332129029357145, P50=0.6039141101757253, P51=0.5766091761798263, P52=1.9262071910289218, P53=0.36300758712817277, P54=0.046162075571514444, P55=0.0034978933737696664, P56=0.0017400163338362688, P57=0.0003667622144864849, P58=9.455163896785472e-05, P59=4.047279732963688e-05, P60=2.9099347122296294e-05, P61=4.3449049553836834e-05, P62=0.00013427692613962253, P63=0.00023119530556996916, P64=0.0014342659928721327, P65=0.009997442560245299, P66=0.06340849177147423, P67=0.18579769337683197, P68=0.12463137492541915, P69=0.05520279566959765, P70=0.07445210203085637, P71=0.16115047222087817, P72=0.40751101546031765, P73=0.4618936816290308, P74=0.3788651877191135, P75=0.4296314444827361, P76=0.41423722125496126, P77=0.4648564924585862, P78=0.4609011400204631, P79=0.5192322748520599, P80=0.5550015451501589, P81=0.19577532361491007, P82=0.03153005968682473, P83=0.010876951925547239, P84=0.0102192465967577, P85=0.009547131199377959, P86=0.009338408397794077, P87=0.016558266139613458, P88=0.03818138601479745, P89=0.0414066714082292, P90=0.06518434095698455, P91=0.13079745208197818, P92=0.34730165551136327, P93=0.4304419863892703, P94=0.4637064581812524, P95=0.6982955131949133, P96=0.9259554334389853, P97=1.1290675974523587, P98=0.34480553543246784, P99=0.30258142047820996, P100=0.12157267827011364, P101=0.07769300929947054, P102=0.06273667396009744, P103=0.06672801873642752, P104=0.0686140678003443, P105=0.18289406858691434, P106=0.49121953552826214, P107=0.3253798997736629, P108=0.41256993419978993, P109=0.4786160342760373, P110=0.4462388450292052, P111=0.7805548893114643, P112=1.027017635674238, P113=0.19397656664770987, P114=0.08923841894707052, P115=0.11956141890805501, P116=0.15880977451312434, P117=0.06939555655676653, P118=0.05872973343760509, P119=0.03108200990314361, P120=0.07337083412231697, P121=0.2043362632334039, P122=0.0448625492437535, P123=0.022206024667336947, P124=0.03280953324502225, P125=0.0122338863665841, P126=0.005759879158097321, P127=0.008634737049067202, P128=0.007414607318844159, P129=0.01296872713187611, P130=0.009272162887450202, P131=0.020357228322037322, P132=0.035799363854881944, P133=0.011375026884834811, P134=0.010807057312306561, P135=0.021229095434108187, P136=0.026056814380560945, P137=0.01132447726901281, P138=0.01864483419318262, P139=0.018547195169432144, P140=0.01975662725413223, P141=0.004912522073990455, P142=0.004937577855014496, P143=0.003633860425369055, P144=0.0022461585365215592, P145=0.0021939912665068697, P146=0.0026344685758220004, P147=0.0026579777422691265, P148=0.005178889226889871, P149=0.006234558428814621, P150=0.0038321555729273382, P151=0.0023157529101196332, P152=0.00081185623075628, P153=0.0006417663585911203, P154=0.00022663072416464114, P155=0.0001248095259766838, P156=8.545663309640378e-05, P157=0.00016729907843070796, P158=0.00016168466359615689, P159=0.0001473502062152524, P160=0.00016448987768275967, P161=0.0001190259260328201, P162=0.00014396056436757033, P163=0.000275122317720114, P164=0.00024209952236459915, P165=0.00029000947424811705, P166=0.00033070792672684925, P167=0.0003438942183061705, P168=0.0002865616959915192, P169=0.00020346586923252323, P170=0.00014762098284826046, P171=0.00016833319072486882, P172=0.00020008600808152683, P173=0.0001975329253619837, P174=0.0002350285963402834, P175=0.0003620923079278585, P176=0.0002064357066526155, P177=0.00013372395557695134, P178=6.988306123956563e-05, P179=8.298447414039534e-06),Row( P0=-465.2603047365581, P1=22.97896046068521, P2=-31.599144413011587, P3=-4.416157911314597, P4=-11.479241071765959, P5=-10.393723510264618, P6=1.8309614859626155, P7=5.946960441701527, P8=17.23494381512339, P9=17.628735570177792, P10=16.373044080171063, P11=23.75409973345935, P12=-0.6145906557386824, P13=-2.3952909253027537, P14=-1.0735459599070942, P15=8.345286912588085, P16=11.565773166286554, P17=7.261991588549357, P18=8.48232230913531, P19=7.5069236020651475, P20=4.445601968101456, P21=8.552659007807904, P22=0.08990393853152472, P23=1.0680872973840632, P24=3.0532921166989975, P25=7.53462604403482, P26=2.9443362030821283, P27=3.2045065977543845, P28=1.3517654746055054, P29=3.9327289278214694, P30=1.9732951751646248, P31=2.9419610084805603, P32=1.2143520416011437, P33=3.568006001072138, P34=1.329851971973311, P35=2.909109563690105, P36=1.6509860630554734, P37=2.5914719753559137, P38=0.37506350832710433, P39=1.8735584380817685, P40=0.5637749938670896, P41=0.5015461408240709, P42=0.4801745320296298, P43=0.481578039306752, P44=0.5106215366818933, P45=0.5394814613995385, P46=0.5407153795130591, P47=0.5928243350700984, P48=0.6910558306009167, P49=0.7346192354034913, P50=0.7460250894244137, P51=0.6681032249010498, P52=2.60319362259408, P53=0.41365997429904583, P54=0.01676033944476404, P55=0.0010809683671966631, P56=0.00023943375085481723, P57=0.00023663267575500186, P58=0.00013169957793774157, P59=5.761420363086358e-05, P60=5.100807898679448e-05, P61=6.611878775984583e-05, P62=0.00010682000680765275, P63=0.00011571287457391294, P64=5.684020631832528e-05, P65=0.00012482156105783664, P66=0.00022296047229131453, P67=0.00028876712988745436, P68=0.0002649562263828237, P69=0.00031783402983647896, P70=0.0006875818440547214, P71=0.007074743242022273, P72=0.024078543043392138, P73=0.038076880328938925, P74=0.1310106360619173, P75=0.2959883960466597, P76=0.4552546412685038, P77=1.0519735313681418, P78=1.461576546731847, P79=1.7765943663534185, P80=2.741236638665093, P81=4.972887408476995, P82=5.118981967801543, P83=2.940047178949806, P84=1.9848695364438325, P85=0.9708069097659378, P86=0.841627510037763, P87=0.7199517419462859, P88=0.17860770144604873, P89=0.011598128484861282, P90=0.002467535352321732, P91=0.004245590291085544, P92=0.01384800492024411, P93=0.18953911231190207, P94=0.519422916788296, P95=0.46915512059776804, P96=0.3514210852764965, P97=1.1754552214157628, P98=2.5241868688400526, P99=1.046062223856002, P100=1.3276236770213092, P101=2.0926127679471653, P102=1.596910159528507, P103=0.6224139412711598, P104=0.6436532328239762, P105=1.335577384474026, P106=1.5749776450236215, P107=0.11476772981051467, P108=0.026235411164019146, P109=0.02877987837840418, P110=0.08832668484316486, P111=0.06524747107689022, P112=0.04932738894217893, P113=0.12312735710733648, P114=0.2715711996019541, P115=0.22641543485615262, P116=0.0893827303529909, P117=0.07820580429023037, P118=0.08624925605646247, P119=0.5909970819757883, P120=0.5161253023069885, P121=0.15458768918351787, P122=0.10060666983828549, P123=0.08347434227033322, P124=0.1295383734950545, P125=0.03954606584664621, P126=0.03549415781226138, P127=0.02597287214688232, P128=0.01089934653027865, P129=0.011381593631278069, P130=0.0064469474408635164, P131=0.010279377253389491, P132=0.01246309756650031, P133=0.008850934588431548, P134=0.011645301677151648, P135=0.017293416764471818, P136=0.05721611344759028, P137=0.03196824566312295, P138=0.019893485723100963, P139=0.019567784634190254, P140=0.0114194298106944, P141=0.015317857965394819, P142=0.009041233837026104, P143=0.007137468192688666, P144=0.008145033065248511, P145=0.010793122407713247, P146=0.008689017996152317, P147=0.007205330807183981, P148=0.012505944392988693, P149=0.011718658634394287, P150=0.00516274937669358, P151=0.0026013119460710035, P152=0.0010941354260717185, P153=0.0009927936095831145, P154=0.001002149495774633, P155=0.00042593214419386335, P156=0.0003076246668856539, P157=0.0004197302194151624, P158=0.0004728282186465979, P159=0.0003835639025460345, P160=0.0006099427199700679, P161=0.0007159014743197984, P162=0.0005696111762611705, P163=0.0006135190468906436, P164=0.0006173574761845072, P165=0.0006065327858740565, P166=0.0007517269620226351, P167=0.0008531452000706017, P168=0.0007337635621167255, P169=0.0006905577726518168, P170=0.0005268594650410401, P171=0.00041565977797456034, P172=0.00046515682820293846, P173=0.0005438442360491702, P174=0.0005382876384168703, P175=0.0003107777429409352, P176=0.0002910631814198907, P177=0.0003155058269185117, P178=0.000204179858473056, P179=3.5778307291102674e-05),Row( P0=-607.3499208230287, P1=33.59679557633879, P2=-24.591081326041827, P3=-6.41749542326763, P4=-21.598006917687226, P5=-17.5432292661805, P6=-10.59389349531358, P7=-8.043600677382663, P8=-4.390828790137725, P9=-16.77934429664176, P10=-5.642395696742979, P11=1.5231230128474667, P12=-2.062565073774804, P13=6.911524939345301, P14=3.6061732134228053, P15=9.664695238290044, P16=5.772317950482823, P17=5.865546546018721, P18=8.098067620094465, P19=4.283860528343576, P20=2.7777814607682307, P21=2.1209890670271303, P22=-0.588469433810154, P23=2.0066843992414864, P24=4.056812787870418, P25=1.2939956014806846, P26=-2.442025240031599, P27=-0.3536897558717665, P28=-2.4189978896330757, P29=0.7770741585426081, P30=-1.069910777909736, P31=2.8334060755379804, P32=0.044210699352387826, P33=0.8565335243725352, P34=-1.3248296396970742, P35=1.8582944733146505, P36=-1.6093303408599098, P37=1.3896569145739517, P38=-1.4126270402002383, P39=0.5030923090246405, P40=0.6888209069799608, P41=0.6322554383990828, P42=0.611712934548481, P43=0.5921205359484307, P44=0.633478066850776, P45=0.6643235801141114, P46=0.667130264307538, P47=0.5853743412052757, P48=0.5901095638130491, P49=0.5851732683613684, P50=0.6505305381821891, P51=0.7517577046890784, P52=4.191376064895014e-05, P53=6.918111392229161e-06, P54=8.746710079351093e-07, P55=2.0880872425210054e-06, P56=8.735738682300576e-06, P57=9.005085518009338e-06, P58=2.9440842359141857e-06, P59=1.3871493838018424e-06, P60=1.414614809998034e-05, P61=0.0003665504392269723, P62=0.006285215209104422, P63=0.00915223301239616, P64=0.0069020557627081, P65=0.023740604026341587, P66=0.11133030322681935, P67=0.18858135300459455, P68=0.11000997255121234, P69=0.06903164866957764, P70=0.054921097854165976, P71=0.029438596098851404, P72=0.037692035992001216, P73=0.040273162524153826, P74=0.04075100055365778, P75=0.034490740713022335, P76=0.01566988454420369, P77=0.022122387293256735, P78=0.02247812060392035, P79=0.027039308334450268, P80=0.0324303032911706, P81=0.048592596087039794, P82=0.060985752825453765, P83=0.039005935909680725, P84=0.02321435275690341, P85=0.02312563855242922, P86=0.042472858877181614, P87=0.04263062503649726, P88=0.058627784996813335, P89=0.038374674869322445, P90=0.019421997999421167, P91=0.03483696625187126, P92=0.06840109297183213, P93=0.046527182360882643, P94=0.044627768411918164, P95=0.07866365232250701, P96=0.032379158640656706, P97=0.011475196829553435, P98=0.04948338027565685, P99=0.12974462843029158, P100=0.03097422892714031, P101=0.009213594595196852, P102=0.005464077347213774, P103=0.005094159614365452, P104=0.004979447833938427, P105=0.0013041771990469349, P106=0.000499110138399229, P107=0.0025786831284931016, P108=0.00467310545400531, P109=0.0010747723628648361, P110=0.002005507406496191, P111=0.0053041006642590765, P112=0.0017959689271924584, P113=0.0015248810459278558, P114=0.010911134797465478, P115=0.008477190653974455, P116=0.004468620513229854, P117=0.003915022251977835, P118=0.0015312786757081278, P119=0.0014699144089070071, P120=0.00277039262960769, P121=0.005832452771102927, P122=0.001260852707232998, P123=0.0017407465374209593, P124=0.003992392540475753, P125=0.005838231988216779, P126=0.002795329838070712, P127=0.0015858943490043075, P128=0.0009241694371986348, P129=0.0006084399405022121, P130=0.00037303219857830417, P131=0.0004082443610520481, P132=0.00035771439142388546, P133=0.00027679158965393324, P134=0.00041221148188454334, P135=0.0005388577798929699, P136=0.0005569998922909751, P137=0.0011346846138378173, P138=0.0012481102892624764, P139=0.0006814378163028055, P140=0.0005592979308715126, P141=0.00032340857150375585, P142=0.00020799820547354086, P143=0.00011071077507243524, P144=0.0002442336531941368, P145=0.0002301569565920917, P146=0.0004719645258181925, P147=0.0004289276339754737, P148=0.0006989315506860189, P149=0.0006089275946447082, P150=0.0006174614595990436, P151=0.0002539849772490692, P152=0.00020045060843588562, P153=0.00012935806199941362, P154=7.74949780768285e-05, P155=6.788735860037365e-05, P156=3.5019813482077826e-05, P157=2.9766006086641278e-05, P158=3.529930908650773e-05, P159=2.8537564998719202e-05, P160=4.9827833331153434e-05, P161=4.9227891160174356e-05, P162=5.860648802322756e-05, P163=5.586598588869102e-05, P164=4.357688547596638e-05, P165=5.050489633637254e-05, P166=7.174717260774758e-05, P167=8.548363463030718e-05, P168=8.776779677332248e-05, P169=5.8960205525254636e-05, P170=6.34732768348575e-05, P171=0.00010753189168562589, P172=0.00013184558520139847, P173=0.00020000387533207538, P174=0.0002503989061821683, P175=0.00017910999467513696, P176=0.00013531376601497944, P177=0.00020193315356356224, P178=9.634373960178208e-05, P179=1.0689460129973122e-05),Row( P0=-611.3550271797662, P1=39.14088869451235, P2=-28.86683913765742, P3=-4.670827697646663, P4=-20.55581005239111, P5=-21.24466935900004, P6=-7.850864904481663, P7=-10.13836523721703, P8=-4.157821338059175, P9=-13.81270673132464, P10=-10.821960923622765, P11=-3.451808427474078, P12=-8.109244635218097, P13=0.4709562150249098, P14=-4.668372938533831, P15=7.123622132158759, P16=5.014790827044809, P17=3.75972722865918, P18=10.19947570918228, P19=3.4613640660984224, P20=0.47140408653871085, P21=5.374201769976065, P22=1.179912253790698, P23=0.12742931184050899, P24=-1.5873834614682334, P25=-3.959071090988938, P26=-7.569123630085838, P27=-1.1987479966367924, P28=-2.1308769133870173, P29=-0.5530838192052975, P30=-5.245170714342941, P31=-3.8843158028846463, P32=-5.033422699567359, P33=-0.7505482188635334, P34=-1.745714747830683, P35=-0.9141481884554375, P36=-6.214988329533293, P37=-4.127390622188863, P38=-2.6665749270201573, P39=0.014215696035056053, P40=0.6394734252848071, P41=0.5980221603724531, P42=0.5810766771550469, P43=0.5943105835430617, P44=0.5802404590233784, P45=0.6024540409267266, P46=0.6005972576077973, P47=0.622534264376936, P48=0.6407093848148859, P49=0.5827762604680755, P50=0.5375638188095866, P51=0.6053007376926649, P52=8.780707230947098e-07, P53=9.652076128470535e-07, P54=7.6746287820698e-06, P55=6.182340234632359e-06, P56=1.1169329139436757e-05, P57=1.0034033323218887e-05, P58=2.9116777838598617e-06, P59=1.976006492289626e-06, P60=8.681933467835942e-05, P61=0.0014066215590507062, P62=0.015137864653660758, P63=0.07161976110245843, P64=0.07616789950474064, P65=0.04123258271473536, P66=0.07813248140071158, P67=0.11363928274535964, P68=0.08113376567024648, P69=0.060696330534532245, P70=0.027979995322392738, P71=0.0064317898427051845, P72=0.002003068205877704, P73=0.0021633589133448136, P74=0.003922297781498985, P75=0.014285183058639586, P76=0.04050480318960791, P77=0.10690203597264906, P78=0.08342860481893741, P79=0.049170179170077086, P80=0.045012573003979615, P81=0.11110741574407863, P82=0.20178999330721073, P83=0.09060759279902388, P84=0.07575250430591667, P85=0.05121087595645928, P86=0.046007085470968635, P87=0.06294509402564473, P88=0.08526943062315642, P89=0.07413150646198509, P90=0.04228967300959332, P91=0.024053759961632958, P92=0.02755678681478113, P93=0.022824056345055323, P94=0.0467597820962118, P95=0.036645645552107246, P96=0.04465713053672689, P97=0.08765904705436034, P98=0.07970093352930911, P99=0.024642829022116685, P100=0.002164143767698684, P101=0.0010475036109168954, P102=0.0010324184063602367, P103=0.0030528148840864517, P104=0.010462969324646219, P105=0.02517105658645893, P106=0.028509675581400926, P107=0.005575516274748042, P108=0.0012591243222690395, P109=0.0018978434248564228, P110=0.005368469913120857, P111=0.003801544873358173, P112=0.02432354318640625, P113=0.023021226274831927, P114=0.01623847512605113, P115=0.00640559548744647, P116=0.0033293074987170517, P117=0.0028346673742618476, P118=0.006191002086260772, P119=0.007039430232646037, P120=0.00477552514371878, P121=0.004091262134316003, P122=0.005227172253146484, P123=0.00614161950494121, P124=0.0029873734583667675, P125=0.0025725500465532468, P126=0.003262769296874485, P127=0.002044342973413717, P128=0.0010393280636946764, P129=0.0010485847599806897, P130=0.0007329697371718982, P131=0.0008421361708594884, P132=0.0010044212394185314, P133=0.001038796719709233, P134=0.001176033234487435, P135=0.001598546254827166, P136=0.002527178270497531, P137=0.003052531880242568, P138=0.0018251397267252274, P139=0.0009268513192578019, P140=0.000574913161624301, P141=0.00042759677620189646, P142=0.0002884189270028767, P143=0.0002161529582635365, P144=0.0003076022075047605, P145=0.0003568265551765731, P146=0.000511085646324306, P147=0.0008095582053710725, P148=0.0010720142519116116, P149=0.0007981121363988476, P150=0.00048598857541230615, P151=0.0003515214389160473, P152=0.00021828263751262207, P153=0.00010665678380074309, P154=7.542832934738535e-05, P155=3.791959351752903e-05, P156=3.080273179355424e-05, P157=2.474045215116927e-05, P158=2.5478981611688457e-05, P159=1.5030546213016296e-05, P160=1.876246514811958e-05, P161=3.5644938470923294e-05, P162=3.580862063014978e-05, P163=4.4476201288561556e-05, P164=3.762738264545759e-05, P165=2.2106616713728335e-05, P166=2.8110473926542606e-05, P167=4.8099717197386666e-05, P168=7.269933556468843e-05, P169=9.373576940525669e-05, P170=0.00013448185266030517, P171=0.00025010260217141276, P172=0.0002600277377516097, P173=0.0001929573208313675, P174=0.0002992895354182978, P175=0.0003159828305571774, P176=0.00039388083540262364, P177=0.000728422734286249, P178=0.0010614006306314523, P179=0.00011625527988187201),Row( P0=-594.5365953771802, P1=32.41087659332672, P2=-22.941153612812496, P3=-7.668870609651127, P4=-19.914093399813815, P5=-17.695345654929405, P6=-8.911571039818783, P7=-6.687491127459515, P8=-5.169583276530448, P9=-13.180413872838006, P10=-3.0612420562441303, P11=4.898583568207989, P12=-1.1793803070376512, P13=6.845506206402797, P14=2.967755071419043, P15=6.325512253969528, P16=4.874083700890209, P17=5.611741781628822, P18=8.059998933427238, P19=4.361128297164152, P20=-0.6554943865123305, P21=-0.23351702173472544, P22=-2.83851470228824, P23=2.9238339173752146, P24=5.093381339215658, P25=1.4692190721053293, P26=-4.068080268797126, P27=-0.3065483819269866, P28=-1.0747774211484995, P29=2.8720118929840357, P30=-0.712047260050171, P31=2.3147898211234685, P32=-1.7225309104351407, P33=1.9778458300767783, P34=-1.4083700509774957, P35=0.43248318438447103, P36=-2.189775912111816, P37=1.4180510004141225, P38=-0.523039313550636, P39=0.5284503977730183, P40=0.7533393198575453, P41=0.7216405387250043, P42=0.6582495890750742, P43=0.599191820061721, P44=0.6418610248113094, P45=0.6969489736588677, P46=0.6863646412988753, P47=0.654925703020809, P48=0.607489556299251, P49=0.6233717042783304, P50=0.6386970622128845, P51=0.731304234841968, P52=0.0003888242541713403, P53=7.619364561905605e-05, P54=8.212138782446815e-06, P55=6.0926653363344185e-06, P56=1.167818404656913e-05, P57=1.9032990626300686e-05, P58=7.390838844402133e-06, P59=2.0516154792711856e-05, P60=0.0003377052841613296, P61=0.0018871849069976411, P62=0.005544922727092366, P63=0.013469204952537749, P64=0.014223083942162653, P65=0.05788178359043633, P66=0.17834626242385582, P67=0.3120892345177675, P68=0.28392733874292475, P69=0.12116209337291761, P70=0.03470828996965492, P71=0.01712143620762594, P72=0.035759645320057605, P73=0.09399068044636684, P74=0.09795312321774263, P75=0.05419801706638802, P76=0.047773814623716576, P77=0.03420596766282925, P78=0.024259612575296862, P79=0.03417458960410273, P80=0.04653816877539772, P81=0.08402506565473715, P82=0.13765177447244367, P83=0.1073774365793887, P84=0.08012009709074117, P85=0.06446768711187142, P86=0.05883155125852282, P87=0.04002942046188131, P88=0.013386444139987224, P89=0.006741159723455831, P90=0.013452830275793685, P91=0.04708996742267194, P92=0.11369625806004864, P93=0.0822388002287458, P94=0.10273871655202894, P95=0.15430828070831515, P96=0.12362244279032426, P97=0.06457135512043306, P98=0.010423235801600306, P99=0.004032360521437214, P100=0.0024711631698002955, P101=0.0037186769067210964, P102=0.0022576046329842087, P103=0.007981106029591213, P104=0.015212642906671153, P105=0.01261941815876783, P106=0.007274342516685603, P107=0.005762811607764143, P108=0.004458575459765547, P109=0.0010715085016420052, P110=0.0007473088094931975, P111=0.0030269504184127346, P112=0.0072171574361927305, P113=0.0031218062523632327, P114=0.0048424131324083805, P115=0.003081964204811186, P116=0.0034263843905835614, P117=0.006055416545777423, P118=0.0074944880421772145, P119=0.001118486774056425, P120=0.0014901030839974392, P121=0.004204728372021303, P122=0.005901938971641313, P123=0.0018813147111413889, P124=0.0025956984926909267, P125=0.007676569370046631, P126=0.0058866512806353905, P127=0.003936600382671282, P128=0.0021176574223343773, P129=0.002359000626986847, P130=0.0008210171107435597, P131=0.00033373354483665264, P132=0.0007084959460583749, P133=0.0009040679599530141, P134=0.00047212319474011284, P135=0.0012511275985060113, P136=0.0018027807507350814, P137=0.001304268279273769, P138=0.0025530061580307187, P139=0.001312176945027052, P140=0.0004936618369997256, P141=0.00027748042352561186, P142=0.0002657796555720848, P143=0.00022474871376921792, P144=0.00022854648125267475, P145=0.00035201119981804265, P146=0.0004551278997192735, P147=0.0003922397098527726, P148=0.0007305906120435501, P149=0.0007907042378075445, P150=0.0004278575915362519, P151=0.00034579360665433983, P152=0.00018694974130234143, P153=0.00013242976499857, P154=8.935354657866488e-05, P155=7.967861602118086e-05, P156=7.308830038281048e-05, P157=8.177123124259276e-05, P158=7.960687174728854e-05, P159=0.00011336920780114129, P160=0.00018227187107218364, P161=0.00019543181677985662, P162=0.00015737179067902546, P163=0.00019118425443736482, P164=0.0001348592434308549, P165=0.0001373659148996137, P166=0.00016917916849711748, P167=0.0001192938056089652, P168=0.0001270294166353406, P169=0.00011730832187733733, P170=0.00011355337852740048, P171=0.00010009839499272816, P172=8.977426207613628e-05, P173=0.00013717686821622103, P174=0.00025457627242587907, P175=0.00022043962425801888, P176=0.0001874645544121223, P177=0.0001837747831069543, P178=6.162897438857117e-05, P179=1.0662352729142865e-05),Row( P0=-555.5600875379627, P1=38.98378362317406, P2=-19.971385196020485, P3=1.488634454753786, P4=-13.915806718625234, P5=-11.097050631933936, P6=-3.5750357409385756, P7=-1.0542976553455938, P8=1.7753545166495353, P9=-4.79746440991337, P10=1.655272977394305, P11=12.49737071140608, P12=6.780616476319143, P13=13.516491210672037, P14=12.041324736681231, P15=11.916854477108012, P16=5.393647016604536, P17=5.212884164992695, P18=8.310586607065252, P19=3.310529816191999, P20=0.10056482265614737, P21=1.2074120246284423, P22=0.9989992769418355, P23=3.06366501723142, P24=3.335415506550209, P25=-0.2732773478771254, P26=-2.956339101052491, P27=3.484486460966609, P28=2.3109771486210566, P29=3.2919745162874676, P30=0.2770473348123965, P31=4.266807433437431, P32=3.0652297510459765, P33=3.9191213304316435, P34=0.549449170528581, P35=2.8872149239078473, P36=3.526414266608836, P37=6.050117613159961, P38=1.4291494681694639, P39=0.912982730024489, P40=0.7092171129333994, P41=0.6932780913856829, P42=0.671199340375145, P43=0.6216330452517846, P44=0.6016766953584788, P45=0.6444488295949257, P46=0.6679736843134666, P47=0.6162502567246984, P48=0.5829165850461249, P49=0.6292468474696774, P50=0.6070072608241582, P51=0.6771371325918488, P52=0.13590244047111022, P53=0.009870308071513796, P54=0.00010027714949067032, P55=2.570701554782753e-05, P56=5.519614073544512e-05, P57=0.00011273853625340795, P58=4.920115111355888e-05, P59=2.3336816837568073e-05, P60=2.858017646446902e-05, P61=3.4213806299343975e-05, P62=5.271788984247433e-05, P63=0.00013325804329449332, P64=0.0003040506805029686, P65=0.004708830663230428, P66=0.030883090927177184, P67=0.06768321523195547, P68=0.08295837434600652, P69=0.16133183406577478, P70=0.17204751616217787, P71=0.0693087645813749, P72=0.05844367172004219, P73=0.12798060339529013, P74=0.16520305752371892, P75=0.10384987536819194, P76=0.08380141148170135, P77=0.34180599942152107, P78=0.5075845590410546, P79=0.1913890022336209, P80=0.01558747305336676, P81=0.004719066825705923, P82=0.016644324840822778, P83=0.05183146689113724, P84=0.07281132884025904, P85=0.06585734830178488, P86=0.07115694432857614, P87=0.11077745203825162, P88=0.08693266375625593, P89=0.03160400796555776, P90=0.01907659111461291, P91=0.026543599242049153, P92=0.05898970081427939, P93=0.06697915433851205, P94=0.1893378578065172, P95=0.2102185877529204, P96=0.0722613947596082, P97=0.010240676729450818, P98=0.014781797844467079, P99=0.06000126862587999, P100=0.023916816303116067, P101=0.0031057239103850654, P102=0.0006298365627286052, P103=0.001068203047762862, P104=0.0031128514901173944, P105=0.00917735723473212, P106=0.050990677351789936, P107=0.1408923504168535, P108=0.09765043679133205, P109=0.01851327706658124, P110=0.0016501018638260024, P111=0.0034534079030547494, P112=0.0018526879061978262, P113=0.001082548371321282, P114=0.0031872543092983, P115=0.026172927408025872, P116=0.030441285097937348, P117=0.0044169071656957, P118=0.0014377062346804458, P119=0.0012316250167559736, P120=0.010420784196219075, P121=0.017770440369217457, P122=0.003655948305772476, P123=0.0025525309555377617, P124=0.002410248513969602, P125=0.008476377704519259, P126=0.013525492393953734, P127=0.003161947666651011, P128=0.0044995783468354976, P129=0.0017383457442457896, P130=0.00045013915267274636, P131=0.0002180826487349485, P132=0.0007126731679475178, P133=0.0012683397808920666, P134=0.0008473107317808522, P135=0.0008742266959975363, P136=0.0027122791059089814, P137=0.0036366374164740114, P138=0.0016547473619547913, P139=0.0010437932992519493, P140=0.0014345362477517925, P141=0.001002685635551141, P142=0.0006388277094113767, P143=0.00042766328562922, P144=0.0008908593276645249, P145=0.00095598042008316, P146=0.0007138973564662339, P147=0.0012731364781452044, P148=0.0029236736253761032, P149=0.0023736417660627277, P150=0.0007201599612357602, P151=0.00037340736057003224, P152=0.00017205613344144287, P153=0.00020360309951963692, P154=0.00017641440427753507, P155=0.00018039351659809212, P156=0.00013156468389489127, P157=0.00011948156422334344, P158=0.0001019039128582952, P159=8.339446398372971e-05, P160=8.25564664405059e-05, P161=5.9506795192735283e-05, P162=9.580791590847642e-05, P163=0.00012200382782964997, P164=0.00019324758020470512, P165=0.0001404936942436626, P166=9.478203479965738e-05, P167=8.339161185518643e-05, P168=9.012887140211763e-05, P169=9.030435337107818e-05, P170=9.408238621382421e-05, P171=6.496770560851318e-05, P172=4.38387925183122e-05, P173=6.244443670579125e-05, P174=5.9764296372802194e-05, P175=8.694409822707602e-05, P176=8.264883243777621e-05, P177=6.874622217889108e-05, P178=4.0474485542528214e-05, P179=3.843303869595821e-06),Row( P0=-577.8134311495561, P1=42.06359252402952, P2=-26.4038390204097, P3=-0.5127256035883179, P4=-17.391265617512964, P5=-13.408132488353663, P6=-6.564075309653678, P7=-3.6630028299144923, P8=-6.575906549680995, P9=-12.76769699925296, P10=-3.643519655513261, P11=6.117241030278508, P12=-2.6235250353167237, P13=6.635124446137381, P14=4.596510795593381, P15=5.563666633088713, P16=4.778645101757774, P17=6.815324636171946, P18=8.430695757427346, P19=4.390092394926174, P20=2.5319703718788538, P21=2.909005139739965, P22=0.12889385021392527, P23=3.2496169206190624, P24=3.877814999919986, P25=-0.02208562684178216, P26=-1.2590374820375871, P27=-0.347400726125807, P28=-0.6245083325636707, P29=3.24607967771226, P30=-0.11237878028953499, P31=3.7994127484723017, P32=-2.5380600487401663, P33=1.0292710980118234, P34=-0.12441634439869044, P35=1.7015698204522867, P36=-1.8473617096078472, P37=-0.26002644207173903, P38=-0.8404547328582435, P39=1.557334353834858, P40=0.7033524759320802, P41=0.6518882551999301, P42=0.6206977186507057, P43=0.6063007125361524, P44=0.6070709960703898, P45=0.6430516338061962, P46=0.6535683363715556, P47=0.6620496024524769, P48=0.631668752097169, P49=0.6180360094285288, P50=0.6058891658369516, P51=0.6567435144152206, P52=0.011769862674701337, P53=0.00032091720073998745, P54=2.0351483749484273e-05, P55=2.264949190721809e-05, P56=3.784250014929917e-05, P57=4.781320050229684e-05, P58=3.381988031140937e-05, P59=2.186462186797577e-05, P60=0.00013299690257887595, P61=0.0012910831321861203, P62=0.006927573843362328, P63=0.01554762487977656, P64=0.01155258162436795, P65=0.015078743436688481, P66=0.054928257094361554, P67=0.1146996584692857, P68=0.06670824161126344, P69=0.04128037198173758, P70=0.09262918734445821, P71=0.09929882664448021, P72=0.06670918516734671, P73=0.04704685538321768, P74=0.056032643191075364, P75=0.08312223232711519, P76=0.0733256792788747, P77=0.10049410830626293, P78=0.05399934669580179, P79=0.025195270397970728, P80=0.02561790829822489, P81=0.03937389683661467, P82=0.05751093337858939, P83=0.1420032955644485, P84=0.20021694385064975, P85=0.15179541788215767, P86=0.029539750602043134, P87=0.010519327396945395, P88=0.01718236709420547, P89=0.014702839952965174, P90=0.015013727518125025, P91=0.019568585607172426, P92=0.027081473633197337, P93=0.025884317899160535, P94=0.05593328449930844, P95=0.09361766201804439, P96=0.06995655089424203, P97=0.03959135690090778, P98=0.023787361621721237, P99=0.01046881702114767, P100=0.003413368983686776, P101=0.007485519321967734, P102=0.0069333159529041295, P103=0.009771036036390292, P104=0.014393355260090742, P105=0.009305831838706013, P106=0.002753974469053864, P107=0.0026882020955688248, P108=0.00451273988272591, P109=0.0054292241746949745, P110=0.004525915122139185, P111=0.010420737150073156, P112=0.01094196498583363, P113=0.004898508559592692, P114=0.003450474557009545, P115=0.0014135634046519737, P116=0.0027037383709208815, P117=0.0016306034893640508, P118=0.0020794096272587752, P119=0.003302403910791938, P120=0.0028196179652758106, P121=0.0028428356775920273, P122=0.003659933292697213, P123=0.0035578658404848917, P124=0.00325182924185837, P125=0.006883474066135253, P126=0.006273035865345163, P127=0.0012898547924367944, P128=0.003292743542245197, P129=0.002704248878129826, P130=0.0007802704184001551, P131=0.0013471246450214459, P132=0.000995332307613223, P133=0.0007379809734028182, P134=0.0012210380685598527, P135=0.0008387493316295322, P136=0.0007603159727032627, P137=0.0006597869841550673, P138=0.0008153683871005558, P139=0.0006337563209119167, P140=0.0008653511973464516, P141=0.0009515377306225967, P142=0.0008713062008872199, P143=0.0006601985923194529, P144=0.0004960820045588703, P145=0.0004193070686955891, P146=0.0003029988195830672, P147=0.0004517059035717103, P148=0.0005292055273965474, P149=0.0006406918541895159, P150=0.000833047029973789, P151=0.0004883830000023221, P152=0.0003713163073550668, P153=0.0002720462760129077, P154=0.00010339295446080288, P155=7.817929875494476e-05, P156=9.782275844590969e-05, P157=0.00015074899666265014, P158=0.0001382063829733086, P159=0.00013784749665059963, P160=0.00013208398158890585, P161=0.00012420862539095333, P162=0.00014807380094273963, P163=0.00013430646843062813, P164=0.00015113513605635428, P165=0.00014639357980528276, P166=0.00012629818881278125, P167=0.0001545712861605427, P168=0.00016871828657144913, P169=0.0002014454086269552, P170=0.00019938331289946288, P171=0.00017568580871943348, P172=0.0001224001656590392, P173=0.0001236942829232026, P174=0.00013910337818313575, P175=0.00013565285340561857, P176=0.00013126031928385607, P177=0.00010179109654650353, P178=5.051457319978559e-05, P179=6.111640137554098e-06),Row( P0=-699.07376260792, P1=47.19197492349384, P2=-12.718664407065994, P3=-2.725511806293702, P4=-15.386000379454911, P5=-17.532939002980708, P6=-11.583571349927443, P7=-9.476962399542957, P8=-5.940429366397538, P9=-10.874750045500212, P10=-8.63196162655519, P11=-3.963858119627981, P12=-5.6956659270325956, P13=-0.3719203242273262, P14=-2.5323833510474367, P15=8.142790864387075, P16=3.743649973085002, P17=4.759979713028625, P18=11.324274617123063, P19=6.515778196433261, P20=3.004285619345691, P21=6.858796716904432, P22=4.372796183419699, P23=0.923690036223388, P24=-2.3266924502520365, P25=-3.8927722025527998, P26=-6.009025312889466, P27=-0.9047588176645178, P28=-0.5280926792078028, P29=-1.157946645328452, P30=-4.104933394520836, P31=-2.656836027959777, P32=-4.4021192360419015, P33=-0.8112293016804174, P34=-4.890368534284646, P35=-2.6318009931968573, P36=-7.924919220342719, P37=-2.258620352236023, P38=-0.1273364863903227, P39=2.145641790473627, P40=0.665739479908174, P41=0.6114616612710773, P42=0.5850624568839653, P43=0.5900757870063732, P44=0.5996025611392177, P45=0.5858930523534618, P46=0.5785164080165992, P47=0.6298239287752223, P48=0.6884742888602692, P49=0.6150787144228991, P50=0.5645984249095164, P51=0.6376313946676112, P52=3.3363788440963545e-06, P53=1.0043921394695602e-06, P54=3.898574100644532e-06, P55=2.8561613111047846e-06, P56=3.180220643531968e-06, P57=3.456314737081875e-06, P58=1.396593623161552e-06, P59=1.0150295181330496e-06, P60=4.167493647688105e-05, P61=0.0015771107980488265, P62=0.009439120887867856, P63=0.024261317189388905, P64=0.05547590165375826, P65=0.038952684818032726, P66=0.023808129460442168, P67=0.03381602670202047, P68=0.02921847718146996, P69=0.020759778375884216, P70=0.010665008364263238, P71=0.0016788476627723071, P72=0.0009729149007372031, P73=0.0018243778499458972, P74=0.002331659276858309, P75=0.0035732958734563266, P76=0.00767174669136784, P77=0.03145824885447777, P78=0.03824765903563452, P79=0.017852014194744702, P80=0.01189717035019435, P81=0.015032184923828731, P82=0.013316773472312861, P83=0.008781673378060009, P84=0.012116447405532372, P85=0.008950410359509617, P86=0.004782070996436917, P87=0.006604059739297327, P88=0.006038890002483192, P89=0.003983488971378025, P90=0.0022832612160110974, P91=0.0009843682564930794, P92=0.001066776087914688, P93=0.0007342202938070928, P94=0.0004842332126321982, P95=0.00048036175289621646, P96=0.002475038946414234, P97=0.0028242944541618, P98=0.002264609075480772, P99=0.0006427031957541232, P100=0.0001064890716714522, P101=7.145977625734988e-05, P102=9.603006133545088e-05, P103=0.00010071388750159862, P104=0.0001446313795873305, P105=0.000598700745321001, P106=0.00047725465501564923, P107=0.00013620357674351338, P108=0.00010298083500067686, P109=9.940746799781358e-05, P110=0.0005020847330813876, P111=0.0005488809667655424, P112=0.0005729339440719363, P113=0.0002840089405988166, P114=9.987244676727291e-05, P115=0.00015836885350179478, P116=0.00022826714068672438, P117=0.0001450826319989084, P118=0.00028370046696338887, P119=0.0007283394730560167, P120=0.0005032761266155019, P121=0.00015476146368200255, P122=0.00014364800423690772, P123=0.0002482135584059523, P124=0.0001250790401987163, P125=9.685374815527554e-05, P126=0.00011910400272718124, P127=0.00010079810298655608, P128=3.9661601660294616e-05, P129=1.669638417299803e-05, P130=1.930170947358328e-05, P131=1.8524690460786664e-05, P132=5.390782672003872e-05, P133=6.48268999154363e-05, P134=4.897584942229802e-05, P135=8.989222962317831e-05, P136=9.463097067655157e-05, P137=0.00017263996649213802, P138=0.00011906959712032501, P139=2.9781738942418365e-05, P140=1.8560675702864147e-05, P141=8.312821958701744e-06, P142=9.470610431998498e-06, P143=9.623202772893561e-06, P144=1.0106838089926485e-05, P145=1.2707596079019813e-05, P146=1.3169921975164146e-05, P147=1.594989855572816e-05, P148=2.8492964283213043e-05, P149=2.8008661869323675e-05, P150=2.0583111400184707e-05, P151=2.4320347464831387e-05, P152=1.7294098400193986e-05, P153=2.6856654472048443e-05, P154=2.250010549339409e-05, P155=1.0486559385349611e-05, P156=8.619127975345644e-06, P157=8.450811404541864e-06, P158=8.10856831219982e-06, P159=1.2248769087995955e-05, P160=2.145839976411096e-05, P161=3.101415810895646e-05, P162=4.4483006947892864e-05, P163=3.4510763971744787e-05, P164=2.0093022689968694e-05, P165=1.4279654280579255e-05, P166=2.3186842033760952e-05, P167=4.206046193636201e-05, P168=7.401712752091548e-05, P169=4.900567524764651e-05, P170=3.111883378304937e-05, P171=6.075242478204807e-05, P172=0.00011163218920930204, P173=7.971736692659301e-05, P174=5.5181285728423566e-05, P175=9.500421508998762e-05, P176=0.00012656644271700514, P177=0.0001758609418585033, P178=8.287611630724599e-05, P179=8.976912287434703e-06),Row( P0=-483.3270252027597, P1=25.54472486210935, P2=-29.007920268668826, P3=1.2382262117175347, P4=-9.983543847706205, P5=-7.080448634009419, P6=3.847208844826585, P7=-1.6826011090305006, P8=6.206689513580349, P9=8.691512122599176, P10=8.75381624270626, P11=17.851090813451986, P12=10.186234510591577, P13=11.524153151431237, P14=4.31700259003661, P15=9.78499277589072, P16=7.066246780065657, P17=6.050468727439727, P18=8.304322604585527, P19=3.6355587609896047, P20=1.8851041996462372, P21=5.319432844862337, P22=2.601610199585534, P23=4.260441136825043, P24=-0.006774808416814125, P25=-2.9391997063477913, P26=-0.1143948294890259, P27=3.203756292390971, P28=1.252105101073499, P29=4.11301318404534, P30=-0.0032732345781513682, P31=3.300400896797423, P32=1.7539706506335702, P33=2.2803424315105314, P34=1.3730484739424886, P35=1.432402314620429, P36=0.9947538774130256, P37=0.6875263776991015, P38=1.3741946104895262, P39=1.755816910676542, P40=0.6232180218552229, P41=0.6715463707161747, P42=0.6975458967338937, P43=0.6353888370589897, P44=0.6459380456210327, P45=0.6506871096104214, P46=0.6528960861033102, P47=0.6887574167562314, P48=0.7349099839129956, P49=0.7199564151731475, P50=0.6675994442830685, P51=0.654602131224035, P52=5.75902761062722, P53=0.5472686328643426, P54=0.028239962932083442, P55=0.0026899085587078738, P56=0.000919745593689558, P57=0.0002328670174536542, P58=8.154711205900824e-05, P59=2.2368206958394395e-05, P60=1.4784621249567641e-05, P61=1.4317392705589518e-05, P62=3.0275549543072707e-05, P63=0.00023211718113617038, P64=0.0020488890128356405, P65=0.005023244261265481, P66=0.01563018487467839, P67=0.04978457102392411, P68=0.12002234512583748, P69=0.15118491165212272, P70=0.15159288871552545, P71=0.14845447219264266, P72=0.19818286125359572, P73=0.3250436681907912, P74=0.3355033793529108, P75=0.3347293409016521, P76=0.32734982665006945, P77=0.6355143761101055, P78=0.610701297552733, P79=0.6353223848925217, P80=0.6664789363038925, P81=0.33130651004219186, P82=0.10592592273413301, P83=0.053961952841625, P84=0.10228350443022323, P85=0.32497867800143804, P86=0.3424651717637908, P87=0.13098371856309227, P88=0.06304104969373796, P89=0.05852290529681612, P90=0.10243149332608968, P91=0.16284376309134416, P92=0.20706355801916168, P93=0.11933862501009032, P94=0.20484792403066968, P95=0.565896144905934, P96=0.5688950358831969, P97=0.7676297039559162, P98=1.0218059978887837, P99=0.3737024550253889, P100=0.08619568776075857, P101=0.11204517886663833, P102=0.11057178663497748, P103=0.12185065552412219, P104=0.09816915809197349, P105=0.11091351061967042, P106=0.25035720904646047, P107=0.8301715551255171, P108=0.8688842888853711, P109=0.5026127132220868, P110=0.26389456640688963, P111=0.1316412465494659, P112=0.08822255897393012, P113=0.0928029236761171, P114=0.12319446095818794, P115=0.11344913347878677, P116=0.2712686929935599, P117=0.29510492703727165, P118=0.1454154347277367, P119=0.13749611191164166, P120=0.12497862153986393, P121=0.0408543664063793, P122=0.0249667780225222, P123=0.01690503451931083, P124=0.04331156117471793, P125=0.02969290421632339, P126=0.003976280609203961, P127=0.005965437026412266, P128=0.024568008329911052, P129=0.013455030966805594, P130=0.008943281907933595, P131=0.010851901083629966, P132=0.030732667249764836, P133=0.018404099319436873, P134=0.020239545471075128, P135=0.02762128865213569, P136=0.01312477707731947, P137=0.02116573104323607, P138=0.028563128725510388, P139=0.020398229193457476, P140=0.024514624649436483, P141=0.010840560678082832, P142=0.004570423768896672, P143=0.002560544025298246, P144=0.003120501071810734, P145=0.0030183014185695555, P146=0.0030018572663758278, P147=0.004150170918319751, P148=0.005054531175526015, P149=0.004900451849692257, P150=0.003603784859494233, P151=0.002526553255721111, P152=0.0012705852548596497, P153=0.0009287329090452197, P154=0.0004196492418439112, P155=0.00020438104907322328, P156=0.0002959620882599133, P157=0.00033683172462352124, P158=0.0003252670496658335, P159=0.0002280922293546439, P160=0.00027325345679784683, P161=0.0003182170329256878, P162=0.0002830249004428259, P163=0.00022840395700758685, P164=0.00020718464314057342, P165=0.00020613667839791507, P166=0.0002608527818597648, P167=0.000267877667933153, P168=0.00023258310769411336, P169=0.00019714330724616785, P170=0.0001584993474220048, P171=0.0001472194758758536, P172=0.00016417554605021133, P173=0.00015828586759583727, P174=0.00021703912495876906, P175=0.0002260539872927748, P176=0.00014893411457442718, P177=9.226394492038706e-05, P178=7.394463796036717e-05, P179=1.3045945467167744e-05),Row( P0=-626.1263775226705, P1=44.46017016865593, P2=-31.069441888440082, P3=3.065666959385684, P4=-15.888911739042111, P5=-12.69551614561073, P6=-5.410854501793294, P7=-8.911417485293729, P8=-2.507599400059183, P9=-10.918494506296955, P10=-7.493756640064103, P11=1.6231646578598915, P12=-1.3435456327749016, P13=4.919724141375814, P14=0.4947521881623146, P15=9.918792796743285, P16=6.658684927397094, P17=6.966346089507169, P18=15.84805506179714, P19=10.130773933588722, P20=5.075825739710861, P21=3.8767947746664375, P22=-2.1799620419073245, P23=-0.7144142503932545, P24=3.156965385148424, P25=4.0299151253992544, P26=2.410775541747946, P27=2.303842930654976, P28=-2.8009411917476763, P29=-0.8268417396376965, P30=-2.63168574718415, P31=1.5020876633625857, P32=-0.17940048644064727, P33=0.6534869563397141, P34=-1.7021953270841017, P35=0.3602770295590515, P36=-1.5034421982063744, P37=2.199952479597912, P38=1.3538371080364981, P39=1.1047900072080934, P40=0.5989186362587222, P41=0.5954386382635474, P42=0.5675403422670103, P43=0.5627953044892056, P44=0.6546824830245994, P45=0.6950262517444787, P46=0.6175852643703674, P47=0.6104099656320884, P48=0.606843631017118, P49=0.6420403564813941, P50=0.6350312559865534, P51=0.6335793098498617, P52=0.00553583738799552, P53=0.00028594329654264416, P54=1.3765479352338514e-05, P55=9.051542776341486e-06, P56=6.784751013858554e-06, P57=1.0115175753508056e-05, P58=5.675395194669208e-06, P59=4.004161375785845e-06, P60=0.00014332923145774537, P61=0.001919625841043246, P62=0.005478299986829741, P63=0.006907717779736933, P64=0.03739277995352145, P65=0.15072890983335369, P66=0.13300754302875992, P67=0.07072267531437974, P68=0.03682875183658648, P69=0.030056240200959268, P70=0.029187668407276816, P71=0.054297127218866745, P72=0.07619643764204707, P73=0.0510871179581045, P74=0.012462494090866463, P75=0.0022328511386131194, P76=0.002233079610301934, P77=0.008093557833994102, P78=0.04412043536978152, P79=0.0905013366108059, P80=0.07791667636456925, P81=0.075048190976114, P82=0.09437187289442764, P83=0.04026445262830873, P84=0.008289372187169916, P85=0.0057601699122746906, P86=0.0034805437919442116, P87=0.003965721581714388, P88=0.00835032795695983, P89=0.008557316599915023, P90=0.03088673955205718, P91=0.08613367139201389, P92=0.0544052021151447, P93=0.029785556557743408, P94=0.036283970700632294, P95=0.03731753855058294, P96=0.01567533578321067, P97=0.009586789116283459, P98=0.007045943450699246, P99=0.017207859535213164, P100=0.011773256479269214, P101=0.006358924758109448, P102=0.008364622934900943, P103=0.009583348786900592, P104=0.00791440717578577, P105=0.005191699885498355, P106=0.010089545914653057, P107=0.009768284681570774, P108=0.008861269842999049, P109=0.019243900866318563, P110=0.028914438669873282, P111=0.02124063885105338, P112=0.013555235718589109, P113=0.006811599125615633, P114=0.00850236677811229, P115=0.016227376112823516, P116=0.01076377577312482, P117=0.009005902510894082, P118=0.002055891790416158, P119=0.00302036637502904, P120=0.007035909009943306, P121=0.014723831660365968, P122=0.0011455427790721226, P123=0.0010891806688501386, P124=0.004060140185585513, P125=0.0065575331156698755, P126=0.0012460487768290612, P127=0.0004429356840552435, P128=0.0012919217468476151, P129=0.0031251289459147957, P130=0.00043531256535854707, P131=0.00035700466939951077, P132=0.0003764970859698658, P133=0.0011213621434869468, P134=0.0009768587284471062, P135=0.0006904102939672787, P136=0.00105287968759667, P137=0.003096976238579413, P138=0.0019502765305843973, P139=0.0009152409455524962, P140=0.0009608389230942425, P141=0.00032190007373631487, P142=0.00021561831573787047, P143=0.0001954532740413496, P144=0.0001571151225877291, P145=0.00021496011378421804, P146=0.0003664227319807969, P147=0.0003089295478718317, P148=0.0005804047022522278, P149=0.0003671125699541821, P150=0.00022672371563941623, P151=0.0001301730579788812, P152=0.00011666391614980659, P153=6.121360074584329e-05, P154=6.306136437562117e-05, P155=4.0243623707046934e-05, P156=2.0652652079887897e-05, P157=3.106998495998465e-05, P158=2.5678354036170786e-05, P159=3.052407139710179e-05, P160=2.4074684740952673e-05, P161=2.3237042532930853e-05, P162=2.852073197808014e-05, P163=2.690315608198008e-05, P164=1.3491934832670987e-05, P165=1.2761417701999593e-05, P166=2.320908924721401e-05, P167=2.9709764275948172e-05, P168=2.045456358219551e-05, P169=1.681091448431607e-05, P170=1.665220789304953e-05, P171=2.167006836432223e-05, P172=2.8600134099827407e-05, P173=1.3552796492742961e-05, P174=1.3474183020727768e-05, P175=2.8632791741926672e-05, P176=2.3454854756993266e-05, P177=2.8817459833574794e-05, P178=2.3830960743178874e-05, P179=4.01405697679331e-06),Row( P0=-636.2386831293685, P1=49.06887954550996, P2=-16.981106593086665, P3=3.2961065020509452, P4=-15.367388605980754, P5=-10.392962410796457, P6=-5.177425342123641, P7=-1.8801128138709229, P8=-1.7788246947743147, P9=-12.372366953606278, P10=-4.658139887001695, P11=-1.7140946908274182, P12=-9.722600413756954, P13=1.350902204355924, P14=-1.7756754813617597, P15=6.102456802072144, P16=5.67850137286158, P17=4.879393089533185, P18=9.188472500077376, P19=3.755200818135405, P20=3.8239899616804007, P21=6.122201511774569, P22=5.921352792222057, P23=4.8011698099758435, P24=5.536134972330521, P25=3.1915691703389126, P26=2.2411741314337315, P27=2.9330814042320177, P28=-0.21968022239241095, P29=3.233071945583905, P30=-0.29220332379942554, P31=3.400856573300405, P32=0.6622514480986396, P33=1.1316525853056063, P34=-2.1011772993143865, P35=-1.687537706529813, P36=-1.3331520142950788, P37=2.814314459104189, P38=0.9985510333240858, P39=-0.03373508339349096, P40=0.7169451921986566, P41=0.6715755426112511, P42=0.602343657058398, P43=0.6092309431915982, P44=0.6545821911470175, P45=0.6469898000711198, P46=0.6138815656784181, P47=0.576968449739683, P48=0.6296772765336287, P49=0.6530653173902865, P50=0.6773592317437878, P51=0.6956132449855821, P52=0.013721873097492093, P53=0.00036135923652663883, P54=9.591548254337867e-06, P55=7.939674924860598e-06, P56=1.9887617782812297e-05, P57=2.0296657491954887e-05, P58=5.624987650791908e-06, P59=5.270277185127952e-05, P60=0.0012050679831508225, P61=0.0061569179301037376, P62=0.03336864893285429, P63=0.04442902825332364, P64=0.02129047041520387, P65=0.038704212745662325, P66=0.0436256763125608, P67=0.03080147522120232, P68=0.0745869739150791, P69=0.0864764143614908, P70=0.01998472709096896, P71=0.0023394773534126468, P72=0.0024505550367208404, P73=0.004453986391266531, P74=0.009510735593281654, P75=0.0070056547055777985, P76=0.002801786894979968, P77=0.004201380518487694, P78=0.012602785507096181, P79=0.01187931031737306, P80=0.0061329832864793765, P81=0.0044459890016948595, P82=0.006058219282139457, P83=0.012223030440467174, P84=0.03063170199344604, P85=0.07471519925096386, P86=0.10671876108924427, P87=0.05136839826039195, P88=0.0049567106864052, P89=0.0012390633746757207, P90=0.0094450023367061, P91=0.013675136519318678, P92=0.009767483096952254, P93=0.005053752627921717, P94=0.011133749094454365, P95=0.006737105388906488, P96=0.0010852123096084987, P97=0.0014898362481936735, P98=0.002163099729096572, P99=0.007494099756675832, P100=0.003879603716922052, P101=0.003564176382344889, P102=0.001221234220974236, P103=0.00025200102752174976, P104=0.0003966077442999458, P105=0.0004459775954579808, P106=0.0024330676913053542, P107=0.0010825748620437484, P108=0.0005955090514930719, P109=0.0002990043099664, P110=0.00014125347651403373, P111=0.0015208398912255696, P112=0.003641003878695313, P113=0.0007492578668229884, P114=0.0005082154265123269, P115=0.000300102920407254, P116=0.0012443104956518997, P117=0.0014574165167376798, P118=0.0002479053247585972, P119=0.00098119189081652, P120=0.001528799651891556, P121=0.0006085083126926464, P122=0.0006963615723811335, P123=0.0011970601507524116, P124=0.0039177942314283015, P125=0.0022882158120471887, P126=0.0004672188802050015, P127=0.0005406365088563105, P128=0.0019346019907887366, P129=0.0015157388988207856, P130=0.00023192117595419455, P131=0.00017242234577018333, P132=0.00033606237274347485, P133=0.0003526108661930625, P134=9.15961399716996e-05, P135=0.00024515484045014874, P136=0.00023346018600561684, P137=0.0001420194560833885, P138=0.0005459214247764818, P139=0.00045922409621110006, P140=0.00010915466850244205, P141=0.000551357568067087, P142=0.00023661564214319755, P143=0.00016366853633052156, P144=0.00026932871816052827, P145=0.00014451905907468535, P146=0.00032476672712693043, P147=0.00023632374436666404, P148=0.0007152399562576765, P149=0.0003539242729370318, P150=0.00036718579032811503, P151=0.000145186409317881, P152=0.00010708333039745208, P153=6.257930841985041e-05, P154=5.2766809755281595e-05, P155=3.4311617189610045e-05, P156=3.0368307086965894e-05, P157=3.95800822691183e-05, P158=5.282723880393697e-05, P159=4.3014145346308216e-05, P160=3.101792770725665e-05, P161=2.8618519490437456e-05, P162=2.5309926543062876e-05, P163=2.992943316413551e-05, P164=3.2076954393618784e-05, P165=4.5881397423354564e-05, P166=5.995513316627795e-05, P167=4.4621000655420805e-05, P168=2.895644029064959e-05, P169=2.8284056248708323e-05, P170=2.6767879514279104e-05, P171=2.163039801189133e-05, P172=2.9218799729878058e-05, P173=2.7456561260469093e-05, P174=4.946532911166999e-05, P175=8.355483323002078e-05, P176=4.880165793746162e-05, P177=4.160026444087752e-05, P178=4.01112422128987e-05, P179=4.532930959119379e-06),Row( P0=-695.1416409402788, P1=47.21086142002322, P2=-11.500247629021143, P3=-7.499256878922072, P4=-15.445042011308606, P5=-14.96487525505746, P6=-13.142947124082395, P7=-11.72038635236182, P8=-8.113592828776229, P9=-12.970654632613194, P10=-11.483466767850528, P11=-10.108501071605431, P12=-9.77082238092927, P13=-1.3601550791986132, P14=-4.488793500178471, P15=8.217643997243464, P16=4.881246857598671, P17=5.889333022001555, P18=10.745405096617658, P19=3.628441868010375, P20=-0.912866357477662, P21=2.6120935405905095, P22=1.9001242456913212, P23=3.129888061198542, P24=2.1398400774784037, P25=-1.7533821862917591, P26=-6.771459361594333, P27=-2.2572454606914367, P28=-1.0734985000310115, P29=1.6057086642530578, P30=-0.5850191432177427, P31=-2.3132397303385606, P32=-3.879920101236061, P33=0.4807748484965615, P34=-2.9308099844283952, P35=-1.0137789463802591, P36=-6.458405087480096, P37=-2.9915065029221344, P38=-2.557845190328535, P39=-0.3415770037526529, P40=0.7134714765019075, P41=0.6222080644832426, P42=0.5954020071903164, P43=0.6036811478508407, P44=0.604866302904792, P45=0.6169340657042144, P46=0.6134690224893967, P47=0.5956556226459404, P48=0.5694607179900192, P49=0.5538410342489117, P50=0.5882706954646226, P51=0.6683528111374025, P52=7.999400115238953e-07, P53=3.634403203929078e-07, P54=1.4688736677467715e-06, P55=2.418916925815762e-06, P56=5.521551642843018e-06, P57=4.269314562596337e-06, P58=1.5340620975666565e-06, P59=1.0485400398936238e-05, P60=0.0007494191749070531, P61=0.004019138661230747, P62=0.028706825554870467, P63=0.0632088644398595, P64=0.023103571193126934, P65=0.01769988571209096, P66=0.037267690776555236, P67=0.06967622592517722, P68=0.04894941182015601, P69=0.011093085104785032, P70=0.0022373737305752222, P71=0.001017869682594694, P72=0.0010176975493883843, P73=0.0009151224764385155, P74=0.0019390811601138632, P75=0.003482782876056841, P76=0.004038677037270892, P77=0.005406794531586963, P78=0.007324690597648545, P79=0.009516331964034563, P80=0.00794390054332892, P81=0.007424480530074513, P82=0.006710537032488596, P83=0.018709173944842006, P84=0.0342646515547787, P85=0.013351378673354172, P86=0.004230209425607276, P87=0.0020848047106052443, P88=0.0018843489486576536, P89=0.0011350301706428156, P90=0.0007413341159564257, P91=0.0009350517734590283, P92=0.001000816145940813, P93=0.0004976908666556373, P94=0.0005050898832497236, P95=0.0009637049021203031, P96=0.004377039315531155, P97=0.0030933416727638753, P98=0.0010401102495235688, P99=0.00015488148290576962, P100=7.351008991789903e-05, P101=6.540373579511646e-05, P102=0.00010803464642686484, P103=0.00019786120459929318, P104=0.0006234076592773512, P105=0.0027893120114308814, P106=0.002236231232646678, P107=0.00023293667185572534, P108=9.449896757151349e-05, P109=0.00011148715273119602, P110=0.00016770848703467932, P111=0.00037400753555736054, P112=0.001823489908042976, P113=0.0003768015441325611, P114=0.00012641124626051677, P115=9.214217038304392e-05, P116=5.615025766932094e-05, P117=7.178728391738734e-05, P118=0.00013874507139601802, P119=6.029116716481815e-05, P120=5.6600147976664444e-05, P121=7.39509021785403e-05, P122=9.982594761318287e-05, P123=8.496386646633181e-05, P124=8.835052152894497e-05, P125=5.427362561679535e-05, P126=4.5983999103808776e-05, P127=6.04179862470906e-05, P128=3.3764333231302975e-05, P129=8.724874801631233e-06, P130=1.56027307660484e-05, P131=1.1603873615604585e-05, P132=1.8350289038406764e-05, P133=2.320436356994287e-05, P134=3.634584603517618e-05, P135=4.149661741607982e-05, P136=5.784018681056961e-05, P137=7.00939863178048e-05, P138=7.355694096734931e-05, P139=3.0099500294416996e-05, P140=1.1685575176313507e-05, P141=7.614176002900276e-06, P142=3.930079181730009e-06, P143=4.704375194031422e-06, P144=5.368644638055122e-06, P145=8.173340985745151e-06, P146=1.1485633557307048e-05, P147=2.2542131701479948e-05, P148=2.955895317720013e-05, P149=2.561313144361417e-05, P150=1.705071279613186e-05, P151=1.843933101178343e-05, P152=2.0345765972713265e-05, P153=2.3417128236745875e-05, P154=1.5197962806538039e-05, P155=9.271728889363767e-06, P156=9.067260112455365e-06, P157=1.0096238891432657e-05, P158=1.863521526018307e-05, P159=2.6308546183915996e-05, P160=3.006696422720764e-05, P161=3.6559354096069517e-05, P162=3.9235687306175765e-05, P163=3.5620038719018295e-05, P164=3.3918115339723994e-05, P165=4.285077983161601e-05, P166=5.297941222782936e-05, P167=3.631032651598409e-05, P168=3.2142782666250316e-05, P169=3.395523878900168e-05, P170=4.104222151770823e-05, P171=5.6643272456973025e-05, P172=6.337170902595003e-05, P173=8.504865154707026e-05, P174=0.0001067400424097331, P175=5.529960622524581e-05, P176=5.511701961341707e-05, P177=5.276402014411564e-05, P178=3.4285620829416346e-05, P179=5.500301012163781e-06),Row( P0=-457.1501597944707, P1=16.62843657810387, P2=-36.104734550679304, P3=-17.48449977098007, P4=-18.78083638044917, P5=-16.589099747220693, P6=-4.169935748931447, P7=5.111490498771229, P8=14.322996726295441, P9=6.50572712167915, P10=4.076885227070814, P11=6.692016350935631, P12=-5.0819107784598705, P13=2.659989905021032, P14=4.255304358848567, P15=7.599145497326467, P16=1.2650550601830635, P17=4.992679283998393, P18=7.087883987784981, P19=4.785327246214277, P20=1.1370009338006146, P21=3.9155603594377104, P22=-0.6474729129393235, P23=1.614195207983532, P24=3.5435892682152312, P25=2.12775021937898, P26=-0.5197337217253454, P27=2.0587059194379562, P28=0.6484287149209396, P29=4.72762548747826, P30=0.0848257412632562, P31=2.859894106363302, P32=2.053360389827869, P33=1.0442232584090712, P34=0.5864152675675814, P35=2.62650098511159, P36=-0.8814456786737708, P37=0.8228832831320964, P38=1.710306675634255, P39=1.2204823283096926, P40=0.6949649537316499, P41=0.6404294625936796, P42=0.6124625436401856, P43=0.5647694251440366, P44=0.5658811058028251, P45=0.5814062337317091, P46=0.5530543145315593, P47=0.6087072610510978, P48=0.5939372059098759, P49=0.6207659988940938, P50=0.6435309026405076, P51=0.688961281549169, P52=0.005154769212336417, P53=0.0008412400765636756, P54=0.00010899923633633748, P55=5.208963869056361e-05, P56=7.999402854171509e-05, P57=6.978939609372819e-05, P58=3.3617156979803695e-05, P59=3.373862999786329e-05, P60=2.9586226605303692e-05, P61=5.348656225216825e-05, P62=0.00026259305461256554, P63=0.0005462712997418653, P64=0.0003540040424104913, P65=0.0003625219066779688, P66=0.0018254525343272716, P67=0.007925988854390963, P68=0.03540313805594859, P69=0.06556528958771266, P70=0.07126072590300102, P71=0.03847340498885588, P72=0.019527086462083856, P73=0.031123485133736768, P74=0.0770267264165725, P75=0.17787503445880118, P76=0.35172971142701376, P77=0.8349588201792666, P78=1.337220740997101, P79=2.2827716632427566, P80=2.1402876691041515, P81=2.1193637309754148, P82=3.381487089461828, P83=5.26952639660507, P84=5.10258497166224, P85=2.470190763514417, P86=2.4584198911575483, P87=2.9666637132597793, P88=2.4775540523691775, P89=0.9482182190274265, P90=1.1117014474016602, P91=3.494211509731674, P92=5.6189435283629, P93=1.819290014249718, P94=0.2166510822589179, P95=0.23846273790258585, P96=0.49490459270512627, P97=0.38908690627560005, P98=0.13726983600359693, P99=0.2717893373426131, P100=0.28655647386010635, P101=0.23757590334515474, P102=0.2876859535447258, P103=0.2736057676783257, P104=0.49913522773766433, P105=0.601225970866889, P106=0.29320314659087043, P107=0.13282283714154006, P108=0.11384869001373299, P109=0.06757533981648026, P110=0.0411751030949554, P111=0.12015258892014626, P112=0.08556835398952115, P113=0.10381467378806984, P114=0.15440562623102444, P115=0.08336658546502002, P116=0.06288190299735666, P117=0.36140062551945573, P118=0.15910492742789129, P119=0.04163080612760923, P120=0.06412193617921574, P121=0.08475600073947899, P122=0.08245061135692505, P123=0.12290425322357147, P124=0.3060148512015772, P125=0.6060656830165158, P126=0.4278213351702046, P127=0.04480644837680693, P128=0.03354147002609187, P129=0.023731979585630527, P130=0.01092632968933072, P131=0.008586086994841098, P132=0.005213413626604513, P133=0.009992651302451512, P134=0.00860868568625161, P135=0.005518241039883084, P136=0.005090434126410298, P137=0.007680435754686711, P138=0.030300340856741124, P139=0.0223078566272786, P140=0.023294868514268673, P141=0.022824353815266, P142=0.0059325294442391305, P143=0.0030438309207450245, P144=0.0023489217604504152, P145=0.004402585582886622, P146=0.007382089430155828, P147=0.00755990849850883, P148=0.004731898444820904, P149=0.0035393509386572567, P150=0.0026356727251953195, P151=0.002997292383620724, P152=0.0020987458754080754, P153=0.000618205773764088, P154=0.00047396597282534146, P155=0.0005840793108331928, P156=0.0005182143736131665, P157=0.0003990407775122985, P158=0.00031193771325441265, P159=0.000371053766483451, P160=0.0003600520315919734, P161=0.0002731492988551557, P162=0.0002946824703497835, P163=0.0003332309719137519, P164=0.00039915877060554264, P165=0.0005388426730102759, P166=0.0005718602346574826, P167=0.0005491040930998679, P168=0.0006098900413334811, P169=0.0007551124675342872, P170=0.0006297442505511046, P171=0.0006725405760307883, P172=0.0005731371620737792, P173=0.0006592355192902316, P174=0.0009815334554391796, P175=0.0010060667860664796, P176=0.0008289870529220075, P177=0.0009713110799906885, P178=0.0006740647528398136, P179=0.00010648013986046773),Row( P0=-694.1171375220171, P1=46.218893600943254, P2=-10.623305618894664, P3=-7.342787859070414, P4=-13.353602753001523, P5=-15.854129504766494, P6=-11.226153886562324, P7=-10.253570462476985, P8=-6.123007430814243, P9=-11.611749281262888, P10=-7.477922478019069, P11=-6.5972655108577305, P12=-1.7955047468345797, P13=3.326335926500379, P14=-0.0662914931907636, P15=12.349356366850015, P16=5.567139512771352, P17=5.783643742633616, P18=10.58630474921078, P19=5.717581076157993, P20=2.069187663289341, P21=3.5755005541646203, P22=0.16141680870156638, P23=-2.1207264631784435, P24=-3.038491055563706, P25=-2.2926961002317636, P26=-3.1129338998389695, P27=0.942690999509374, P28=-2.014016806288698, P29=-1.9233510148047734, P30=-4.582830957933858, P31=-1.5728940449810291, P32=-1.7101703412631382, P33=-0.32960783945406585, P34=-6.211507712818287, P35=-2.566138036973534, P36=-5.745821369406342, P37=-1.130412413552512, P38=-2.526113646931198, P39=0.33695255135210017, P40=0.592214097715943, P41=0.5878570059034032, P42=0.6027804593935109, P43=0.6375264974651182, P44=0.6388047836508317, P45=0.6364866475547455, P46=0.564768723526643, P47=0.5267984673003614, P48=0.5784191118364232, P49=0.5805628309345982, P50=0.55266526025379, P51=0.5906177896721075, P52=2.5313099763931385e-06, P53=7.798441519753667e-07, P54=2.1926993699905643e-06, P55=1.8509537452845742e-06, P56=4.404129379166007e-06, P57=4.452532799667458e-06, P58=7.82151480081911e-07, P59=1.6295191150395507e-06, P60=8.495155940529565e-05, P61=0.0017365653240116903, P62=0.005564389570605752, P63=0.016805545580130542, P64=0.05680713236375823, P65=0.04241846535459436, P66=0.03545573835560345, P67=0.050949821592785306, P68=0.03790657571068789, P69=0.05449616450985243, P70=0.04255946334762328, P71=0.012613189895576615, P72=0.001794146095539162, P73=0.0012612075996676735, P74=0.0007544881277828052, P75=0.0006833127017807462, P76=0.0016419035683938087, P77=0.010748775376378094, P78=0.01250249227984222, P79=0.01011739332711097, P80=0.0083636299139934, P81=0.007564605124624015, P82=0.00832255790768809, P83=0.009518717993093395, P84=0.007333369146713946, P85=0.004052066897109351, P86=0.016307558425638947, P87=0.014712537836586088, P88=0.011523524336710443, P89=0.003417526439115692, P90=0.0012336671844446815, P91=0.0007959308507015265, P92=0.0007353097708609857, P93=0.00039783037866014084, P94=0.000448355685935628, P95=0.0004092301765991903, P96=0.0002271926982896182, P97=0.0003425705727109517, P98=0.004979062435648666, P99=0.006321206103182325, P100=0.000981674625757569, P101=0.00022829172096626484, P102=0.00011701920703892734, P103=0.0001302017091134467, P104=0.0002442012784800546, P105=0.00029227364076944956, P106=0.00014222354975495092, P107=0.0016100767694075253, P108=0.0029865266492381075, P109=0.0006777904804021641, P110=0.00012377881537866704, P111=0.00019930340325144744, P112=5.896515586418248e-05, P113=4.9982629695499604e-05, P114=0.0035760038289857905, P115=0.0035076078009533937, P116=0.000714032286621715, P117=9.887961003339977e-05, P118=5.315603476549099e-05, P119=0.00032173528629421514, P120=0.00022192916370903583, P121=0.00013407298284834378, P122=7.683420060155872e-05, P123=7.176233172789526e-05, P124=5.5531206425655135e-05, P125=4.85528969769485e-05, P126=6.799330717535028e-05, P127=4.00901082006511e-05, P128=2.448272812430504e-05, P129=1.2844642839760198e-05, P130=1.8776285091114122e-05, P131=1.892160231042035e-05, P132=4.060231032326251e-05, P133=4.5083695543482885e-05, P134=4.425985870984439e-05, P135=6.103989477623625e-05, P136=8.24321210094685e-05, P137=0.00010708599141159672, P138=6.810074896152061e-05, P139=4.017585322298102e-05, P140=1.504128283258769e-05, P141=1.1369280646110061e-05, P142=7.222445054471699e-06, P143=6.694089852027366e-06, P144=7.606587618726068e-06, P145=8.173818456982041e-06, P146=1.2352623666853492e-05, P147=1.2530940726152703e-05, P148=1.7467571395585185e-05, P149=2.235332225146558e-05, P150=1.4976727866963062e-05, P151=1.390598565917496e-05, P152=1.6255955939973134e-05, P153=1.9875610022627097e-05, P154=1.64835213976585e-05, P155=1.231092355127298e-05, P156=1.591486587723879e-05, P157=9.039217660781927e-06, P158=1.3243335643250766e-05, P159=1.8242474392075606e-05, P160=2.1592947297690322e-05, P161=2.122536704004582e-05, P162=1.9065461092139315e-05, P163=1.6436525653232083e-05, P164=1.2861128285466862e-05, P165=1.3308528397091649e-05, P166=1.9185000760686334e-05, P167=2.2728002230291313e-05, P168=2.7575560631245217e-05, P169=1.861572852601489e-05, P170=1.9724854774631518e-05, P171=1.735298686126736e-05, P172=2.3851701439497912e-05, P173=3.644315520910821e-05, P174=6.147489463402494e-05, P175=7.80208141756368e-05, P176=7.452280131769426e-05, P177=8.79853091907272e-05, P178=4.938766062264271e-05, P179=6.7015670769158255e-06),Row( P0=-615.9590747062015, P1=34.79602173896082, P2=-25.1063986975454, P3=-6.069383796559257, P4=-21.82348738304669, P5=-20.976232992668578, P6=-11.389479453421297, P7=-12.040529166430131, P8=-6.1452017702282395, P9=-16.324446832970715, P10=-12.413751610793819, P11=-5.91294950096039, P12=-6.410965214675337, P13=1.5702494352815852, P14=-3.757096132704053, P15=9.739340687718945, P16=3.320122813402464, P17=4.528337703936272, P18=8.46829244552369, P19=4.850475534423679, P20=0.46675929771860875, P21=2.7208999037887582, P22=-3.4991267656493026, P23=-1.484902747664914, P24=0.0773057055064516, P25=1.7026810856210903, P26=-1.7310559778740564, P27=1.2077404295699052, P28=-3.363116272596175, P29=-0.5066017790148358, P30=-2.720193389946751, P31=-0.7489976263236992, P32=-3.823583777011585, P33=-3.805330968483057, P34=-2.288958429700987, P35=2.1280335166270015, P36=-3.237838211236995, P37=-2.827579877915622, P38=-1.276207128926384, P39=2.0658811857685278, P40=0.5997206727707342, P41=0.5819821569278621, P42=0.6591194378236872, P43=0.6874895713528393, P44=0.6548131746873526, P45=0.5938509425498943, P46=0.6216181586177414, P47=0.5810368888090119, P48=0.5614307787649073, P49=0.604797055304221, P50=0.6389984156806843, P51=0.664750732899592, P52=4.243031179483346e-06, P53=9.052360708053951e-07, P54=1.6431311783314563e-06, P55=3.0112826230451617e-06, P56=6.401568049254074e-06, P57=7.349297211728753e-06, P58=2.7631187258404192e-06, P59=5.558939120877737e-06, P60=0.000426358537534637, P61=0.0036784268762443686, P62=0.010608139726245686, P63=0.026648156816248547, P64=0.038363185709155846, P65=0.17332688925561274, P66=0.1647029265387636, P67=0.09029588459782978, P68=0.03418548012468259, P69=0.032048186021925225, P70=0.07655869994697939, P71=0.05502098644665025, P72=0.0190416170771804, P73=0.004748822634454647, P74=0.0013652092992421053, P75=0.004360936154572071, P76=0.008637229304195661, P77=0.013454748389886067, P78=0.05221798281744568, P79=0.15971558788899787, P80=0.10146011612798454, P81=0.0397137659962213, P82=0.05665765543503877, P83=0.040735536844843705, P84=0.026204459045468104, P85=0.018167247235516337, P86=0.010406724239495988, P87=0.01789873910270731, P88=0.028219971904169937, P89=0.019976928005077443, P90=0.031073753522320443, P91=0.06305957898160326, P92=0.03384438886366976, P93=0.011244436494353425, P94=0.048481929274749494, P95=0.0596295945076071, P96=0.030318492136849538, P97=0.019131990195328286, P98=0.010008647719180284, P99=0.005521974151176287, P100=0.002539104957596516, P101=0.0029947389278398707, P102=0.0033520651559661952, P103=0.017133044479878498, P104=0.012007919555202046, P105=0.0014797627267760312, P106=0.000565083637041195, P107=0.0009304645125301619, P108=0.0015798188546190543, P109=0.004098681504384233, P110=0.017602799596351026, P111=0.01179175188595304, P112=0.0025353714280708646, P113=0.0017544778981531464, P114=0.0018047346643331528, P115=0.004985420806389047, P116=0.012300840239234365, P117=0.011799861700172413, P118=0.00596566798102707, P119=0.0021541636352829, P120=0.0010440508226381832, P121=0.0019167797234441962, P122=0.0021898447650600954, P123=0.0010341357602448057, P124=0.0020690578496566094, P125=0.0025088618784889233, P126=0.0007337254155094765, P127=0.000561200322353411, P128=0.00048635175152635297, P129=0.00016507958888105375, P130=9.681065711210625e-05, P131=0.00024402328134307056, P132=0.0005573803909182307, P133=0.0002998878660725933, P134=0.0005054639336871811, P135=0.0008509488972617197, P136=0.0007168883454350433, P137=0.0015776724488995178, P138=0.001105920649559586, P139=0.0009945583253788848, P140=0.00026801520044596855, P141=0.00014870797293741391, P142=9.177578106033272e-05, P143=8.618187949791764e-05, P144=0.00012923028089491644, P145=0.0001197043674039311, P146=0.00023784281305633387, P147=0.00032685800248753873, P148=0.000569793582359685, P149=0.0005459575156407541, P150=0.0002868858404557839, P151=0.00019448766805960104, P152=0.0001492654104815821, P153=0.00013111434872206874, P154=9.806676242784371e-05, P155=7.836490479133545e-05, P156=3.478259821250734e-05, P157=3.551555524922458e-05, P158=3.815170902354265e-05, P159=2.4620125121751405e-05, P160=2.902441186705629e-05, P161=4.037974332402382e-05, P162=4.926939231744765e-05, P163=4.657406893742031e-05, P164=9.247256117351254e-05, P165=5.7242987668469605e-05, P166=2.6429920986780763e-05, P167=3.082848719573214e-05, P168=4.667201151098316e-05, P169=7.233064601555502e-05, P170=9.280464249630949e-05, P171=0.00010616255405062934, P172=0.00011460441476824838, P173=0.0002494886993911575, P174=0.00029698962412587506, P175=0.00022147559710878557, P176=0.00022548781770367543, P177=0.0002484681562155958, P178=0.0001011327414883884, P179=9.46711614340194e-06),Row( P0=-429.5086401347036, P1=12.073015390685814, P2=-38.92630884464601, P3=-10.29652607640289, P4=-12.917840144911006, P5=-10.107441247584038, P6=3.2458452001825053, P7=10.694225598969982, P8=21.222849592647457, P9=14.916836618146622, P10=4.221134422149515, P11=6.248679699255797, P12=-6.616375456037685, P13=1.3524050580537832, P14=7.2694722148222555, P15=11.660772687476301, P16=4.277491277922044, P17=4.135286223772071, P18=10.343818692146836, P19=7.3316784737821035, P20=0.7011885986891784, P21=4.676062093839363, P22=1.6002065578898095, P23=6.319047584491608, P24=3.156615282810189, P25=3.2403941365710778, P26=1.4583872698442213, P27=4.741806473587047, P28=2.594825670617898, P29=5.397498828106004, P30=1.6910477227078342, P31=3.7065436475946085, P32=3.487728208371723, P33=2.9803325015342828, P34=0.8715975354650375, P35=5.0185853510047425, P36=1.0508500933179308, P37=-1.5686693521681834, P38=0.38597492204875594, P39=3.60254368131467, P40=0.6098514403644832, P41=0.5780846007449548, P42=0.5620351841218136, P43=0.5238730711775281, P44=0.4747584869933676, P45=0.4635708095030246, P46=0.5091763992857654, P47=0.5309383286974643, P48=0.5730922908211087, P49=0.5871255319091702, P50=0.6395415542162354, P51=0.6647619709650167, P52=0.2351113539783567, P53=0.023037936919658882, P54=0.0016637629264347662, P55=0.0008011423513692802, P56=0.00038004657276459296, P57=0.00015405179704950546, P58=6.399348932115092e-05, P59=0.000520525601688515, P60=0.0017367590523378242, P61=0.001039448195289361, P62=0.0003589727232991048, P63=0.00013894415185318563, P64=0.0001065366268802898, P65=0.00016329263995238521, P66=0.00030991856650530237, P67=0.0004828683425089223, P68=0.00046621527777728383, P69=0.0004610585584508994, P70=0.0005532447452128847, P71=0.0006049614392205987, P72=0.0006184658620125938, P73=0.0010164422109366566, P74=0.007373694119800639, P75=0.02899715144711516, P76=0.05325353355181202, P77=0.12566938527457838, P78=0.253212479066114, P79=0.3471327584189297, P80=0.41645180584881175, P81=0.9938506956317468, P82=2.3636478226561453, P83=2.1285619246897403, P84=2.7385533692424096, P85=3.3353073253249628, P86=2.9654984508556383, P87=3.9994740032202456, P88=5.408652891044164, P89=4.476519173013283, P90=5.374960256309563, P91=4.502943263770053, P92=1.583379890683029, P93=1.6112069949039902, P94=5.931602292277644, P95=10.119117924388243, P96=3.311139832495043, P97=0.9107750242604679, P98=0.47947409519081735, P99=0.2234462203516158, P100=0.10736506440494749, P101=0.18717970862524363, P102=0.3074839264519277, P103=0.9554798519978046, P104=0.5187846175193072, P105=1.2138525526434547, P106=1.8108027532135673, P107=1.8763521769028464, P108=1.0866679663131746, P109=0.6909663576125784, P110=0.8549689430967676, P111=0.7326136224677006, P112=0.2859870710223714, P113=0.07650591978212093, P114=0.07040763616715405, P115=0.1500201049344249, P116=0.2049846273138371, P117=0.2836408007505519, P118=0.3999567169175754, P119=0.26742542364528926, P120=0.1398036990360705, P121=0.08504574944906496, P122=0.15716310896736627, P123=0.23140976440504665, P124=0.4943855604171382, P125=0.1672251210076197, P126=0.042299315939219824, P127=0.01544685731426832, P128=0.008709499274399343, P129=0.02106216237385507, P130=0.0881324355173859, P131=0.25297291477981976, P132=0.2777891537688312, P133=0.008889551142939389, P134=0.006904801857010987, P135=0.0075884725964273595, P136=0.022441406803017174, P137=0.047448562104782786, P138=0.04634250969052697, P139=0.0782767763004225, P140=0.09149917613092916, P141=0.015335308319441044, P142=0.0121430107390707, P143=0.006383837600550035, P144=0.00805284766427867, P145=0.012343891071262592, P146=0.01326807449562184, P147=0.007257238689735801, P148=0.007563413202862515, P149=0.011432439908964056, P150=0.010330809025987878, P151=0.004603510133424957, P152=0.003011662910259584, P153=0.002978121012299814, P154=0.0017390488249758658, P155=0.0011679626065490432, P156=0.0008770503977047591, P157=0.0010477769532154946, P158=0.0007079135017475597, P159=0.0008410852549592491, P160=0.0006969529038049299, P161=0.0007409024812254789, P162=0.0008562920427619504, P163=0.0006899095993824782, P164=0.0007763995366001605, P165=0.0012986557057724137, P166=0.0015303736562076134, P167=0.0012194436821894815, P168=0.000817093133012606, P169=0.0007193259904401204, P170=0.000630412875363105, P171=0.0005625654859069419, P172=0.0007178814168544062, P173=0.0009150943368928565, P174=0.0008401579709590677, P175=0.0006410223221269609, P176=0.0006551652434352286, P177=0.0006356229258107022, P178=0.0004132431722192262, P179=5.533945061761692e-05),Row( P0=-544.032128155913, P1=30.829236786525744, P2=-25.747340364347146, P3=-8.954856357085637, P4=-19.109723155441266, P5=-16.585099222182393, P6=-5.73429442540053, P7=-5.866449733776909, P8=-5.6736750308309665, P9=-15.410063988296544, P10=-2.049997177862777, P11=4.088462560964648, P12=-7.098821158579297, P13=5.372420567503906, P14=-0.677441815041996, P15=4.901414407087447, P16=7.164693434187047, P17=5.387682985225403, P18=11.72460560930817, P19=4.326133119755804, P20=4.913233103534959, P21=3.8016447353025224, P22=-3.484694615311048, P23=3.0655694274398133, P24=2.3199232019192277, P25=2.9473553855455377, P26=1.7832183800354728, P27=-0.16822407238839238, P28=-1.9894364740561727, P29=0.13836901309250732, P30=0.3455969835924699, P31=4.450626751886379, P32=-1.7475903578704803, P33=1.2582026061118363, P34=-0.7900323394292864, P35=4.0126368089735704, P36=-1.3995922419854196, P37=-0.6959806923799066, P38=-2.2957991299803693, P39=1.3251500736656108, P40=0.6615066104890297, P41=0.6226881021361942, P42=0.612433517703804, P43=0.6536580920795215, P44=0.6707842177406195, P45=0.6482119228681362, P46=0.6609443684589145, P47=0.60341527982293, P48=0.6103841075106646, P49=0.6030303712315876, P50=0.6522101919193969, P51=0.6885661821781105, P52=0.0013680316519328547, P53=6.244011996991935e-05, P54=4.663290554909288e-06, P55=3.50501152479506e-06, P56=8.928377845330592e-06, P57=1.4714342362288057e-05, P58=7.858402601940697e-06, P59=3.616369223383862e-05, P60=0.0010645826688773436, P61=0.006168656415368387, P62=0.024521178423877216, P63=0.035394950542774084, P64=0.017894725338636917, P65=0.09665073010009671, P66=0.29089126116409214, P67=0.21609451874797472, P68=0.08029224754521858, P69=0.09177111594019978, P70=0.12728848866830467, P71=0.08533017468690803, P72=0.03500633096375088, P73=0.02542004016889879, P74=0.06984075354778191, P75=0.12745046121586995, P76=0.05219416174744499, P77=0.031590525887007025, P78=0.06324298470299465, P79=0.10012235146390924, P80=0.10822351080838644, P81=0.2383167411876255, P82=0.3510703639892289, P83=0.21229852986523648, P84=0.07730981772269602, P85=0.047364652539775225, P86=0.024089452294384844, P87=0.02550251194082421, P88=0.059305526801669214, P89=0.17298979212294852, P90=0.6661146987627787, P91=0.7885814829297103, P92=0.2561625395765789, P93=0.1347832421736454, P94=0.2550537904833452, P95=0.3235553840216289, P96=0.15355584341348336, P97=0.051683533809039214, P98=0.059200895775849034, P99=0.07666961043207299, P100=0.05233968315571937, P101=0.0450739487303272, P102=0.04409784058161198, P103=0.06715023632980899, P104=0.04300729478778803, P105=0.006224647298138023, P106=0.014960243222383937, P107=0.01419734072985594, P108=0.013124379777731418, P109=0.006114606810504571, P110=0.01591206634167594, P111=0.05286173330436793, P112=0.024722576695888523, P113=0.006366080358667708, P114=0.0028579216979330393, P115=0.004744109943909975, P116=0.013997732997244241, P117=0.011325883378768472, P118=0.0032502620085941426, P119=0.005774631943882994, P120=0.028300318609357618, P121=0.02008866170186908, P122=0.013614624494558603, P123=0.018967173785005557, P124=0.04667694346131544, P125=0.02979006669984273, P126=0.005635888633136378, P127=0.012303754882553268, P128=0.01990594263356228, P129=0.027428613849836102, P130=0.004143917999141785, P131=0.0030980689005650924, P132=0.0063672380705059, P133=0.004840948246613276, P134=0.0023216113620152753, P135=0.0019603864882320574, P136=0.002890578493731568, P137=0.002606506321116933, P138=0.0011696143352999521, P139=0.0030875702708389584, P140=0.002514637298242082, P141=0.0005368634066498923, P142=0.0006365308289575533, P143=0.0007709019629415926, P144=0.0004444374098472153, P145=0.0008021867727363626, P146=0.0005507208542767623, P147=0.0013378876553242415, P148=0.0018638179858739225, P149=0.0021354428298538176, P150=0.002787308458483019, P151=0.002475713243363547, P152=0.001585557732286591, P153=0.0007546972175363663, P154=0.00021313607131377078, P155=9.641353306576438e-05, P156=7.362083531921223e-05, P157=5.95042728327468e-05, P158=9.861743760085835e-05, P159=0.00011567775582850842, P160=0.0001786154097962949, P161=0.00035085730066628375, P162=0.0005392155435322986, P163=0.0006522981194247248, P164=0.00046275017884465024, P165=0.000377989841156019, P166=0.00033374292104531124, P167=0.0003722123615072687, P168=0.0005052971063283875, P169=0.0007326178862447199, P170=0.0008758635459271252, P171=0.0007506684647270307, P172=0.000575325849557729, P173=0.0005291504335987073, P174=0.0007035794984895649, P175=0.0007391116042400044, P176=0.0006606971178744975, P177=0.001104088635163074, P178=0.0005651198246219801, P179=3.116455190789756e-05),Row( P0=-547.7784937618435, P1=43.21121311434488, P2=-18.35181511492737, P3=-0.7238086679872805, P4=-14.465766023592693, P5=-10.295434519957718, P6=-3.25614730562711, P7=1.0107522446644237, P8=2.6701959647868345, P9=-2.6003861546303395, P10=6.48877651597742, P11=12.305791363313718, P12=7.94676145426775, P13=12.817952050437608, P14=7.731204836888239, P15=10.05753324430685, P16=7.927273141322407, P17=9.504623229686011, P18=11.26142474913877, P19=7.739976632347428, P20=4.467229486008366, P21=6.496040315715267, P22=4.370157181443914, P23=5.671678994785565, P24=3.5144759440979985, P25=2.206448589408895, P26=1.4920055577804396, P27=4.702975238239911, P28=2.8838869336515014, P29=4.063796708591556, P30=0.29858148399103485, P31=4.9034379427299415, P32=1.7391277708110446, P33=2.7546451610352816, P34=-0.5559779044310094, P35=2.873899764812891, P36=0.7961886877615898, P37=1.944589752521534, P38=-0.6097234735525919, P39=2.6661345019558635, P40=0.671974000783516, P41=0.6729997132399982, P42=0.6630657691046399, P43=0.6596019860952728, P44=0.629291665029389, P45=0.6712307263125488, P46=0.7115837845325841, P47=0.6922980418524288, P48=0.6506731677733438, P49=0.6726696658964084, P50=0.6906089722613216, P51=0.6820265547922861, P52=0.08589619566391926, P53=0.009771233184877163, P54=0.0007579064362942873, P55=2.7959106131976466e-05, P56=3.4258343403288434e-05, P57=4.47921774357244e-05, P58=2.1496285353893305e-05, P59=1.4552293466867534e-05, P60=2.8579629518703183e-05, P61=0.00013497425864083553, P62=0.0010737349703170626, P63=0.0034110986511125376, P64=0.00637308655057123, P65=0.03358520794057129, P66=0.06493883339059327, P67=0.08450986714207828, P68=0.09475407422203315, P69=0.1422084234443033, P70=0.22852494611325055, P71=0.17726869110511032, P72=0.0838176100315844, P73=0.0644159496017377, P74=0.10636869325293193, P75=0.19377781117149367, P76=0.2177448089732165, P77=0.19462961255799124, P78=0.16905060638864558, P79=0.29217591477770455, P80=0.5370582936749687, P81=0.3938195783368315, P82=0.11277201396513675, P83=0.032862853398584135, P84=0.0361156198099374, P85=0.05667651395183368, P86=0.0645273783995516, P87=0.04668274676968563, P88=0.0529084102180466, P89=0.05074585897484243, P90=0.04326607228411115, P91=0.027308332033896973, P92=0.05373353632274654, P93=0.05999136351816645, P94=0.12095674118905884, P95=0.14080802387091126, P96=0.13759175558118797, P97=0.12845050966358337, P98=0.061876886925306185, P99=0.019181520950983758, P100=0.007716854253978555, P101=0.011336440754745835, P102=0.013650011159958514, P103=0.003982700313699509, P104=0.0035573289940212236, P105=0.007179462457664645, P106=0.030637218395498957, P107=0.039911975363383576, P108=0.029264128097473207, P109=0.03665287646962891, P110=0.03981207931308907, P111=0.013699908072646186, P112=0.0015905619319981448, P113=0.001152916597349841, P114=0.012531569937164005, P115=0.04741955820044799, P116=0.03645385131016047, P117=0.003574355212685043, P118=0.00405704145169423, P119=0.0018719562454863217, P120=0.0025072851810334085, P121=0.0049882902392610114, P122=0.018387851364152677, P123=0.04339507946835118, P124=0.01616215447724256, P125=0.004366913756481192, P126=0.004863105658722897, P127=0.001857175620721208, P128=0.002922483484160414, P129=0.003031323555895155, P130=0.004940357530202099, P131=0.0032399293958013703, P132=0.0013322743093352237, P133=0.0009727846186154689, P134=0.001161957488618725, P135=0.0035166242843999384, P136=0.0029704065634035227, P137=0.0010700164533781055, P138=0.002399246495671118, P139=0.004339676137984113, P140=0.0046321688097367315, P141=0.0009397418418519947, P142=0.0009288992029212981, P143=0.0014967179122444365, P144=0.0014035660510203901, P145=0.00068821307337159, P146=0.0007625254966634142, P147=0.0013101418304825021, P148=0.001890163898429242, P149=0.001079286022974775, P150=0.001409969246196035, P151=0.0006512448469331413, P152=0.0006750551478797095, P153=0.00039183418858428613, P154=0.0003030289568711365, P155=0.00023695423729312148, P156=0.00032763328993782936, P157=0.00024302035910439112, P158=0.00024736039585145905, P159=0.00023470452463949523, P160=0.00017709685838640372, P161=0.000103163424437553, P162=0.0001437278379293542, P163=0.00014537394823489175, P164=0.00024951492002378485, P165=0.000207675605296447, P166=0.00018417522313424764, P167=0.0001337848854997591, P168=0.00013087638840826569, P169=0.00010852239594175635, P170=9.46556533648728e-05, P171=6.509542867829654e-05, P172=5.1621389907985055e-05, P173=4.457820853780283e-05, P174=6.65500640821197e-05, P175=5.9165974577771356e-05, P176=7.946106740627138e-05, P177=6.6287982377063e-05, P178=2.9458804443161998e-05, P179=6.606749147979918e-06),Row( P0=-635.0435729563685, P1=64.26239984480824, P2=-22.511370040973755, P3=5.38034787170603, P4=-19.661584792868553, P5=-12.669855894551622, P6=-8.820140391705248, P7=-5.319920337750855, P8=1.1305736074789157, P9=-10.835163903157415, P10=-7.73817910332707, P11=0.812944525440536, P12=-7.080384948478164, P13=2.2356581160069426, P14=-3.3714796526688118, P15=4.791189185863649, P16=2.8082656673008053, P17=1.4151481604048903, P18=8.999399316129596, P19=3.8884904880097, P20=4.579429157965828, P21=4.3537383884182095, P22=3.499524282316698, P23=2.7691039906397927, P24=4.570785946393114, P25=5.638537003611145, P26=4.33922053567516, P27=5.386192230800139, P28=2.59438393507021, P29=3.817413183078459, P30=-0.8570086299235345, P31=2.6116341240186203, P32=0.5894370017353079, P33=1.2230706046162128, P34=-3.1889063903406463, P35=0.4626268149068407, P36=-2.524895342329275, P37=1.4538163530979915, P38=0.1613091423553737, P39=1.615260318695103, P40=0.6405558727621552, P41=0.651439007532183, P42=0.648504244817177, P43=0.6995040628697667, P44=0.7290585558872481, P45=0.7174974329378931, P46=0.6775480316736395, P47=0.6290222863170741, P48=0.6341533234389192, P49=0.6616836155750693, P50=0.6370246885860495, P51=0.6493267987698348, P52=0.0007352169569902341, P53=2.89313318694658e-05, P54=4.769002862051494e-06, P55=8.136937585964841e-06, P56=1.3705845051493332e-05, P57=2.113862320804823e-05, P58=9.041400361488305e-06, P59=7.967972030889201e-05, P60=0.0026542096564323428, P61=0.021091017824646015, P62=0.053844609038863664, P63=0.05286741711916519, P64=0.026069853935292913, P65=0.03212735471381236, P66=0.0405002510101122, P67=0.03831454652413409, P68=0.02797716123311103, P69=0.023393844356765912, P70=0.019725057400406604, P71=0.010518503042678973, P72=0.009598282059235975, P73=0.009914678847895262, P74=0.009322848804589643, P75=0.012373422159977252, P76=0.009538858788762164, P77=0.019311197567128832, P78=0.037543463167584244, P79=0.0362558390250202, P80=0.025097535478335792, P81=0.02151501970981583, P82=0.021123668285893498, P83=0.021727124918654583, P84=0.02660334876347281, P85=0.027004267479444267, P86=0.025473754985637042, P87=0.022224113514990703, P88=0.022172208352743007, P89=0.014077329342633855, P90=0.020908342996309084, P91=0.031184199919112615, P92=0.03861748670475324, P93=0.016574033769757947, P94=0.012508495834501743, P95=0.00558597619701834, P96=0.003301315582317367, P97=0.0019622932026258175, P98=0.01398141104970974, P99=0.025500341603393373, P100=0.009486210811940987, P101=0.010043042038415701, P102=0.008703597761283524, P103=0.003992692621905192, P104=0.0012613765828823383, P105=0.001987991819204622, P106=0.007587258107507813, P107=0.0036938018925291695, P108=0.0030030110512111415, P109=0.0027469767067199195, P110=0.002207579322108354, P111=0.007042909359164977, P112=0.007256431999852657, P113=0.0057320938127973595, P114=0.006780968574094523, P115=0.006004800268138112, P116=0.004072688877296767, P117=0.0029059302655805824, P118=0.0016577060648498686, P119=0.002500314214280431, P120=0.004000830708381295, P121=0.001455679818204352, P122=0.0017887729652409808, P123=0.0017911197642348296, P124=0.0015173814040800398, P125=0.0011023862254406606, P126=0.0006024001957634586, P127=0.00033160528917387333, P128=0.00042126591534623743, P129=0.00017822388771962955, P130=0.0001644869071831334, P131=0.0001833602825266639, P132=0.00017435494868335973, P133=0.0002376254282010559, P134=0.00024516452148515397, P135=0.00016305580048071728, P136=0.0002332462936789321, P137=0.00028298624904926124, P138=0.0003356476090253185, P139=0.00029859747223795425, P140=0.00023767910453087134, P141=0.00016603541807363515, P142=0.00011336364368548376, P143=0.00010094880240524382, P144=0.00011760412231146657, P145=0.00013421406832726755, P146=0.00017561410462083832, P147=0.00029564065724125877, P148=0.000490209166116634, P149=0.00044887356745549744, P150=0.00019442323930079242, P151=0.00010554257614548294, P152=8.032987674546557e-05, P153=6.313346832497744e-05, P154=3.78737805891026e-05, P155=2.4688477007700725e-05, P156=1.7943337481860404e-05, P157=1.7530945443728704e-05, P158=2.2062706968934132e-05, P159=2.273540483099609e-05, P160=1.779470990382554e-05, P161=1.2520432394678168e-05, P162=1.9551658157834218e-05, P163=2.2349292468151638e-05, P164=1.1228606204277153e-05, P165=1.2962517890601029e-05, P166=4.083636217686693e-05, P167=4.613624479725137e-05, P168=4.5561735421134966e-05, P169=2.5306003084157744e-05, P170=1.8213425311249764e-05, P171=1.8174850108631245e-05, P172=2.672401956847402e-05, P173=2.9572993953225722e-05, P174=3.98619130489324e-05, P175=3.518286489457787e-05, P176=2.197206131510999e-05, P177=1.790501942980115e-05, P178=1.0949630065324963e-05, P179=1.8091653999071942e-06),Row( P0=-691.3429062032275, P1=45.4991873969367, P2=-14.7059305542405, P3=-4.580725223052616, P4=-15.053467426033583, P5=-18.621861462462125, P6=-11.898600053179239, P7=-10.254466776080067, P8=-7.602254342033684, P9=-11.571060939139386, P10=-12.082494826116411, P11=-7.697338878475008, P12=-10.027399008839566, P13=-4.452861853086221, P14=-4.925755125235341, P15=6.199847124927167, P16=3.9871750307093015, P17=6.824911161536437, P18=14.260007462074142, P19=9.157298520011368, P20=4.221044000281288, P21=3.457288312357816, P22=-0.602578059200872, P23=-2.9520268617989367, P24=-2.3394486669667924, P25=-0.3229563728612426, P26=-1.6884297536954047, P27=0.9483539244792496, P28=-3.428175954106186, P29=-4.527260642728687, P30=-4.328845668143265, P31=-1.6764619933866751, P32=-3.626620887302827, P33=-2.0657070334702268, P34=-6.175924834136203, P35=-0.6715770362711823, P36=-2.798408451853861, P37=0.27633598100616424, P38=-2.082893857720293, P39=-0.760271225909193, P40=0.6009717564396385, P41=0.5823945777954115, P42=0.6197997922912797, P43=0.6147900210080575, P44=0.614109900233656, P45=0.60237349341266, P46=0.5657576113294859, P47=0.5611808654863281, P48=0.6665662368100743, P49=0.6932088014543453, P50=0.6967479671927691, P51=0.670404495140467, P52=8.895863882728748e-07, P53=3.449733207952441e-07, P54=2.0280797031889327e-06, P55=2.1052143185595206e-06, P56=2.9729783535383067e-06, P57=2.0029248427840133e-06, P58=5.396149181279709e-07, P59=4.9724450360479085e-06, P60=0.0004923659485453305, P61=0.004396753399209801, P62=0.008163421531335063, P63=0.011584717801739919, P64=0.046913388879230085, P65=0.0888442650253677, P66=0.09369671911123215, P67=0.042721181825332355, P68=0.010988048747625025, P69=0.022338155230417017, P70=0.027073804461889603, P71=0.005946220522136265, P72=0.0013795868734754622, P73=0.0012427856981674008, P74=0.001211030354606808, P75=0.001321625081015364, P76=0.0018189777679679636, P77=0.010739761791492717, P78=0.025466057826954693, P79=0.02163618097666672, P80=0.019343481152251, P81=0.02837775385990039, P82=0.016135151358278615, P83=0.007904454203365274, P84=0.004709407039477631, P85=0.0022673680347904245, P86=0.0012232076428604776, P87=0.004454181971315459, P88=0.008509369689075343, P89=0.0030781722970983525, P90=0.003942742075180453, P91=0.0036700066308793427, P92=0.0029976491099045046, P93=0.0074982821073566385, P94=0.005805147664921679, P95=0.0019325845961180655, P96=0.0003887991331202626, P97=0.0002592729264347567, P98=0.0009630514456970769, P99=0.0022304413138604824, P100=0.000759776033456743, P101=0.0005760394252140735, P102=0.002940317073224871, P103=0.0012429426649980405, P104=0.00023291695506462198, P105=0.00024244613922439422, P106=0.0005552382933299226, P107=0.00020621784611348488, P108=0.0003575365999005039, P109=0.0014232946342759112, P110=0.0004551868508380275, P111=0.0003238695518543295, P112=0.00016603806065111683, P113=9.439727204516299e-05, P114=0.0005246259107872631, P115=0.001038116604097509, P116=0.0003975481241995338, P117=0.00027226532901456597, P118=7.441670213207126e-05, P119=0.00021469781706503574, P120=0.0010223004397981942, P121=0.0008159407565795974, P122=0.00010712356055412593, P123=0.00012180203448533765, P124=0.0004347734164840078, P125=8.300089867004849e-05, P126=0.00010687907133455738, P127=6.987638592250499e-05, P128=2.8920586595161386e-05, P129=1.1136277589138048e-05, P130=2.8609193327286326e-05, P131=1.9899673573552404e-05, P132=1.900762979820871e-05, P133=4.8537599967693493e-05, P134=3.932397922679751e-05, P135=6.39169259083446e-05, P136=7.861612831753072e-05, P137=0.0001086295486231736, P138=7.719663158577212e-05, P139=3.316339000243431e-05, P140=2.0649898217402104e-05, P141=1.3205865165366862e-05, P142=1.003402636640998e-05, P143=1.3869423557003112e-05, P144=1.0276977265243003e-05, P145=1.8311074649475045e-05, P146=1.9029682410059623e-05, P147=1.4082816345698737e-05, P148=1.5826047307150795e-05, P149=1.6371015359285444e-05, P150=1.2854937703292717e-05, P151=1.4905638460251672e-05, P152=2.3512831427658584e-05, P153=2.865936911679285e-05, P154=1.9596359016290136e-05, P155=1.2318036772640459e-05, P156=8.909967098380545e-06, P157=7.776358208144082e-06, P158=1.3267585369916844e-05, P159=1.5659283172146478e-05, P160=1.8666127656091427e-05, P161=2.2232525412027782e-05, P162=2.134236244665965e-05, P163=2.843017154741002e-05, P164=2.751477458441452e-05, P165=1.9769482877302617e-05, P166=1.595172996749879e-05, P167=2.712549045916932e-05, P168=4.219278173027825e-05, P169=3.396726515304839e-05, P170=2.973697039691895e-05, P171=2.5443017185972948e-05, P172=4.0014096273880944e-05, P173=4.8105639144826705e-05, P174=5.5327999790246415e-05, P175=4.4849856240553026e-05, P176=3.894273122355949e-05, P177=4.051206782588879e-05, P178=3.599487908800975e-05, P179=6.130612534104943e-06),Row( P0=-497.97283755897735, P1=13.607628327841127, P2=-43.789655300520785, P3=-9.913899635833644, P4=-18.294808072072392, P5=-18.21061889351812, P6=-0.5559426307098018, P7=-2.7258214002703727, P8=8.366400064753394, P9=9.666158355647104, P10=5.844588735351295, P11=12.579675661239829, P12=-3.248468019005692, P13=2.076157585396632, P14=-0.8883894477618685, P15=9.547972945471873, P16=10.818220303197874, P17=7.926639991783978, P18=8.461484754963521, P19=2.3543625246680464, P20=0.842195298780366, P21=3.423849773173644, P22=-3.25736919557332, P23=2.5960074443376993, P24=3.5932106800448156, P25=2.2578358625290607, P26=-1.8865569323044526, P27=0.7782971258574379, P28=-1.5107991134232883, P29=3.01271778275955, P30=1.2489297550745184, P31=3.696792652831553, P32=2.263174891330079, P33=2.227003933399234, P34=4.184415610411624, P35=4.543535328010651, P36=1.090803355699527, P37=-0.6788182304164876, P38=1.262180489478226, P39=2.6155209214029584, P40=0.7263656727239531, P41=0.6091834565434809, P42=0.5673510786791695, P43=0.5289469405033773, P44=0.5337596714948568, P45=0.5425779929400482, P46=0.5188760816642085, P47=0.5443447171065867, P48=0.5448256739804764, P49=0.5760269710149716, P50=0.5810587832257369, P51=0.6651102516520894, P52=0.01167212714033809, P53=0.0009666503513729247, P54=2.7699154208577552e-05, P55=1.0140714292686781e-05, P56=1.8508474039312794e-05, P57=2.3270254987008903e-05, P58=1.2369242745041102e-05, P59=1.0533517393688584e-05, P60=1.1547158554697346e-05, P61=1.6327746361008632e-05, P62=4.0861420110017955e-05, P63=8.16089969925975e-05, P64=0.00031122366239344284, P65=0.010418707077263816, P66=0.06052036271634057, P67=0.11995454452489318, P68=0.050882061919737244, P69=0.011151590258598148, P70=0.01289533996447903, P71=0.02421359536012557, P72=0.08304098592150942, P73=0.14574066114282463, P74=0.17615484434651674, P75=0.1625176137949908, P76=0.21996467254703894, P77=0.8408871955661413, P78=1.0160105770576437, P79=1.0389335647493274, P80=0.7116421106368196, P81=1.1655858788695728, P82=1.9156806387635916, P83=1.8993684514368891, P84=0.984356716408248, P85=0.4244748350182909, P86=0.4371605378826611, P87=0.8106701971704023, P88=1.6399086817881978, P89=0.5504731720482876, P90=0.20854663676784915, P91=0.38222839070508746, P92=0.5225166179075103, P93=0.22703696830081446, P94=0.21016485093730464, P95=0.465236164472064, P96=0.6094835996056319, P97=0.8125809555402366, P98=0.8091780628205281, P99=0.7907129671507059, P100=0.3864447017809761, P101=0.4471683314477867, P102=0.32532973465690274, P103=0.49168067994757064, P104=0.9398496553592616, P105=0.4982020858570351, P106=0.1661804209597543, P107=0.2008028543726724, P108=0.14656366326887227, P109=0.13900449847912644, P110=0.42376626071463197, P111=0.7187391520189904, P112=0.3046710824251934, P113=0.23554311366161204, P114=0.2924276652545636, P115=0.19774369871123976, P116=0.18712558382615796, P117=0.16140387689882638, P118=0.0945236918065447, P119=0.15508058565778385, P120=0.23322983936312233, P121=0.13799954591945404, P122=0.09357465763122272, P123=0.026320178257939322, P124=0.022227473012203576, P125=0.11337232496981696, P126=0.14057112233942434, P127=0.015691838011077073, P128=0.006343492599447881, P129=0.026067477922120608, P130=0.019432117012045683, P131=0.025562565672147326, P132=0.05959024685973391, P133=0.05966528683402409, P134=0.0032451040036061657, P135=0.017142525718836843, P136=0.039059680517514635, P137=0.04666755505891648, P138=0.061281386061416664, P139=0.01813297057794262, P140=0.014617536708758379, P141=0.01077297300506933, P142=0.008200406222387172, P143=0.006541439861527284, P144=0.0042522377497054556, P145=0.0033759213239925117, P146=0.005666429290483394, P147=0.013909530452682888, P148=0.00444756327322011, P149=0.00475276402062489, P150=0.005407514739260239, P151=0.003600248683784613, P152=0.0012216241626853016, P153=0.0005398074566578164, P154=0.0002687819519531535, P155=0.0002167630831282796, P156=0.00014212751184598864, P157=9.314539590393885e-05, P158=9.356783943063497e-05, P159=8.526368347733141e-05, P160=9.256143543632508e-05, P161=0.00015978952238703155, P162=0.00028968482122609327, P163=0.0003565525573359941, P164=0.0004151531972159603, P165=0.00029961751582166225, P166=0.00033189900736542004, P167=0.00029662762817823826, P168=0.0002369685826473248, P169=0.00021036089728857147, P170=0.00022533181238805638, P171=0.0002483851954639754, P172=0.0002571377832935693, P173=0.00023828807207144695, P174=0.0002809792730080531, P175=0.0003066593669182449, P176=0.0002412804351193499, P177=0.00033033670234068385, P178=0.00017599639613976513, P179=2.7598979732455868e-05),Row( P0=-572.0184361081324, P1=29.618811786708367, P2=-30.67744137318041, P3=-2.7020558613628376, P4=-16.91553049000152, P5=-16.16881578166085, P6=-6.686484121395904, P7=-9.81193378345996, P8=-5.547016408864862, P9=-10.764412733532973, P10=-8.581219476493558, P11=6.468232105768665, P12=2.5891140023172716, P13=15.203134538194446, P14=12.032933840886841, P15=15.240867128983606, P16=7.396339524274438, P17=-0.057332302454441354, P18=3.2985063157130456, P19=0.4831856875794374, P20=-0.7971493808433476, P21=5.564150826332396, P22=-0.8767278735035049, P23=-0.022788709845266455, P24=0.4661157546763478, P25=1.6591718322104652, P26=1.9337794295917046, P27=3.6209835796622785, P28=-0.5872420118377888, P29=0.04945133777357232, P30=-0.09316169894851957, P31=0.8981821203124908, P32=-0.10820420506597894, P33=-1.0266851821830019, P34=-1.6849012155975982, P35=-0.5208987429304492, P36=-1.7093219706585612, P37=0.7163208191509787, P38=0.47478310570906634, P39=-0.1493613084110307, P40=0.5921923905295432, P41=0.5975782054287698, P42=0.6446363159963147, P43=0.6422347056155032, P44=0.6577820120434723, P45=0.6921330219285248, P46=0.6121958898888586, P47=0.5642304703741438, P48=0.5315405006139511, P49=0.5352992895096723, P50=0.5171120419562079, P51=0.5653945855385863, P52=0.0003001703224583885, P53=6.089196186977243e-05, P54=8.500945257918368e-05, P55=3.1645940951914685e-05, P56=7.354550391480157e-06, P57=8.26182750708186e-06, P58=3.920384673828664e-06, P59=2.4768570315209943e-06, P60=9.3941448208016e-05, P61=0.0016718451243282009, P62=0.004537590124080181, P63=0.004290650906699015, P64=0.0030080517847993777, P65=0.02494910984816973, P66=0.07665393798073014, P67=0.10699819960614704, P68=0.18703232165517433, P69=0.5178499480693188, P70=0.6309821093775997, P71=0.29440358407278633, P72=0.14743288313168426, P73=0.11103803557139648, P74=0.06353914756710788, P75=0.04287013267762684, P76=0.010728875702588412, P77=0.0032333959661502393, P78=0.0038591295032304965, P79=0.00514001746123468, P80=0.00507505426852895, P81=0.005789422630521333, P82=0.00976609889422521, P83=0.014230972215284285, P84=0.021527632788537213, P85=0.057504283980907005, P86=0.2710360159481953, P87=0.3606941348397705, P88=0.3603109466910163, P89=0.3304657782825369, P90=0.22041893287715886, P91=0.2226661099324067, P92=0.23422398617752851, P93=0.0949712980252816, P94=0.11876313594376348, P95=0.17726335117919353, P96=0.048116193017275245, P97=0.021584249269735765, P98=0.16711388030810978, P99=0.29362207636840193, P100=0.05810513436412675, P101=0.06767671854080719, P102=0.04907239367724098, P103=0.039266781659085405, P104=0.04596451314689776, P105=0.0134923420615841, P106=0.015731193676553953, P107=0.043749958248947986, P108=0.040992751192914824, P109=0.007861097882317748, P110=0.05176570324116986, P111=0.04775243177931288, P112=0.023176619024629284, P113=0.022723290283825828, P114=0.02833898169408304, P115=0.034468030436106065, P116=0.026383400566572835, P117=0.02779186698694124, P118=0.014862595010684364, P119=0.013533195807887491, P120=0.11930071468434066, P121=0.0385368922228913, P122=0.006771741955609946, P123=0.0054195444786811235, P124=0.020181136411626638, P125=0.059991307531961975, P126=0.009741688695448973, P127=0.002832701608782567, P128=0.002907895935687204, P129=0.013069530123986369, P130=0.0029238730435498096, P131=0.0005215739299479847, P132=0.004894644049611367, P133=0.008543398873458037, P134=0.002522596501424231, P135=0.006937300411999617, P136=0.017677817122486082, P137=0.005601947955185595, P138=0.014867134828114326, P139=0.012491411602054256, P140=0.002983021731873842, P141=0.0019142783001125062, P142=0.0011904432713241192, P143=0.0007293492093188711, P144=0.002092704837293619, P145=0.001464416007555987, P146=0.004814419010342745, P147=0.0035350042585371813, P148=0.004556130020911583, P149=0.0024026619006098085, P150=0.001083970544363797, P151=0.0006651180757086544, P152=0.0007583618818844559, P153=0.0002660886678098006, P154=0.00024786011772319255, P155=0.00010230992721420147, P156=6.153437016346573e-05, P157=5.632787280504925e-05, P158=6.445433669710613e-05, P159=3.510240576815162e-05, P160=2.893903038077532e-05, P161=6.698774073743234e-05, P162=8.589001752995009e-05, P163=7.039952440896729e-05, P164=7.845469165071262e-05, P165=5.416556148619751e-05, P166=5.165797248112855e-05, P167=5.3384059426891816e-05, P168=4.4798987205479494e-05, P169=5.0234099463335274e-05, P170=6.689767791243598e-05, P171=6.052115007843939e-05, P172=5.24416117166081e-05, P173=6.916364745273058e-05, P174=0.00011938755077379157, P175=0.0001151495975200322, P176=0.0001030550261869416, P177=0.00011670700076804422, P178=0.00015228089346287137, P179=3.018783491921673e-05),Row( P0=-455.82434277102385, P1=12.358450069723007, P2=-40.08727999795098, P3=-11.57558406955216, P4=-16.04398722853264, P5=-12.49491574636677, P6=-0.5958498368230608, P7=9.832045151625183, P8=21.679879837110352, P9=15.679432405706828, P10=2.575399643535679, P11=4.450262206827975, P12=-10.659251731504037, P13=-0.800491999159336, P14=6.530519377610044, P15=11.600681838281426, P16=5.625978975343076, P17=3.5900138561185115, P18=8.304644181963935, P19=7.629166102068903, P20=0.8425046144488546, P21=1.595102184921874, P22=-1.608559150144312, P23=3.447549428613309, P24=4.020450799471174, P25=3.2483247367729713, P26=0.40526178526613116, P27=2.139617075694026, P28=0.6788408208687245, P29=4.502484304733244, P30=0.013694026730097057, P31=3.29051306237884, P32=2.30996028566788, P33=3.1309467218677374, P34=2.1618331261110413, P35=2.8176447790425634, P36=-0.709106976046737, P37=3.0496951601378455, P38=2.879788955368725, P39=1.3317335690075212, P40=0.6634208333342495, P41=0.6351742955028074, P42=0.5724012940391738, P43=0.5298022542928155, P44=0.5653561809659825, P45=0.5502372964892015, P46=0.4952858048716321, P47=0.5103498725136708, P48=0.5171464651540921, P49=0.5768930896487149, P50=0.600142065884315, P51=0.6598546566866895, P52=0.17353760994571793, P53=0.01497510386821829, P54=0.00029996896466001014, P55=2.4720510009822298e-05, P56=6.0002369049290665e-05, P57=0.00011425981677165743, P58=7.078614589328307e-05, P59=2.7376923881819376e-05, P60=2.8723319398637698e-05, P61=5.630166245003361e-05, P62=9.359210126505066e-05, P63=0.00012956688665253827, P64=8.873495736053985e-05, P65=0.00011122248221863479, P66=0.00024291677676000522, P67=0.0003743921828046269, P68=0.00029850407917087304, P69=0.00033051050561198116, P70=0.00044313142923939857, P71=0.0004054871387232367, P72=0.0005375683031924549, P73=0.0017598852124397404, P74=0.022112624062069784, P75=0.04619046799911882, P76=0.03071790383441407, P77=0.09123530334087042, P78=0.3273169927826873, P79=0.6317909894868993, P80=1.6176021727755818, P81=2.691468286150765, P82=3.771473198915033, P83=3.6865578018989664, P84=4.107590580345253, P85=4.9444626649950285, P86=3.608993474721363, P87=1.8123195172874649, P88=1.3356335028067556, P89=1.0649280266610948, P90=1.2278370801232323, P91=3.294486266395122, P92=4.546938107689093, P93=1.4666957755057626, P94=0.19408206190180316, P95=0.05065134922429024, P96=0.006410141956471097, P97=0.0014642258681282553, P98=0.08321927523848703, P99=0.529232547635839, P100=0.14986106818990338, P101=0.16775707479193652, P102=0.2038600861505864, P103=0.12415876895654211, P104=0.2278818659439057, P105=0.6368395761255818, P106=2.1953878638407476, P107=3.7670801125807674, P108=1.6298129534756296, P109=0.4223899330283346, P110=0.30569378666520025, P111=0.671333718290482, P112=1.0596087833855152, P113=1.6000906479406107, P114=1.583110412708446, P115=0.34903639652400675, P116=0.14386684953541082, P117=0.21572503882970895, P118=0.26646322353900226, P119=0.1174702165587565, P120=0.16140977140590598, P121=0.1713267129163154, P122=0.048390800674895096, P123=0.08170748260861418, P124=0.09114990095471669, P125=0.0815058890234988, P126=0.03649634901660622, P127=0.026254726457687182, P128=0.03218035948152511, P129=0.017699397749736532, P130=0.01954351456263018, P131=0.005491088883006672, P132=0.006008189196227738, P133=0.01943704166738399, P134=0.014999123279570417, P135=0.022340224408947264, P136=0.014269111126235733, P137=0.007238352722261651, P138=0.03592232079109794, P139=0.030155231963144853, P140=0.05219433726510261, P141=0.027755979592565254, P142=0.014074603873381794, P143=0.0045047160684153235, P144=0.008433406822289503, P145=0.01069692710084678, P146=0.00967386891395642, P147=0.013489082498821931, P148=0.009991497317816398, P149=0.005307904161840013, P150=0.0029640188272758387, P151=0.0027976383040044816, P152=0.0007923788344677185, P153=0.0006848095318276001, P154=0.0006733947023395154, P155=0.0005223903418743636, P156=0.00028453343905321583, P157=0.00031226406519984723, P158=0.00030856043057301684, P159=0.000350947878310497, P160=0.0003468345804253493, P161=0.00026866144150167653, P162=0.0002868233742845605, P163=0.0003097843722308918, P164=0.0003014759065769111, P165=0.0004679636510716584, P166=0.0006558270860997312, P167=0.0006461897046277513, P168=0.0005288258155159666, P169=0.0003802313034506231, P170=0.00035296969050266824, P171=0.00032359901385857536, P172=0.00034042975778950953, P173=0.0003462458092479243, P174=0.0003635978649805637, P175=0.0005066668882269198, P176=0.00041224597843471703, P177=0.0003182550404026473, P178=0.00021265624658682082, P179=4.4171913239635045e-05),Row( P0=-648.2366180825217, P1=52.754208319136715, P2=-14.775445358530579, P3=-6.149999130965645, P4=-12.499649290528444, P5=-11.297634632522957, P6=-8.786153330314454, P7=-6.172720613267469, P8=-5.531969755480077, P9=-16.406090655598916, P10=-8.377919898277355, P11=-4.513509459859086, P12=-9.005649164655111, P13=-2.4271745612410838, P14=-6.80488394302842, P15=4.24872375712029, P16=0.2575816723788904, P17=2.810357413782774, P18=9.164192878915795, P19=1.4428846752020115, P20=3.340975929755057, P21=3.0440946031031464, P22=0.9294680168104182, P23=1.7529050087530151, P24=3.027756347861795, P25=-0.4983775608688836, P26=0.10021286277615324, P27=0.27616516562566407, P28=-3.2684160543162775, P29=-1.4197353108402526, P30=-4.316017326434862, P31=0.13579712537180502, P32=-2.984154312983251, P33=-1.6210795078779192, P34=-3.0110349920823962, P35=-2.663704550450732, P36=-4.5104964148091184, P37=-3.199245072999853, P38=-1.9021184120355266, P39=-0.7136239852425461, P40=0.6992305458893003, P41=0.6632599308867536, P42=0.6744735274199127, P43=0.7098158486488624, P44=0.736792282042836, P45=0.7365011301082813, P46=0.7108759353174398, P47=0.6783201309785423, P48=0.672555394969206, P49=0.650246236040251, P50=0.682082264086745, P51=0.7302947315957147, P52=4.874177918564683e-07, P53=4.842879995928015e-06, P54=1.1731296996894177e-05, P55=4.502862121583722e-06, P56=1.2173331619334884e-05, P57=1.3730926080369853e-05, P58=1.4096651188631763e-05, P59=5.1908316688182966e-05, P60=0.001127563307231444, P61=0.008585158877909787, P62=0.02131196400378762, P63=0.021922304821814932, P64=0.012934672968798234, P65=0.02348816382913798, P66=0.05875174504703941, P67=0.11464258796628476, P68=0.06813106874577765, P69=0.008295050166889622, P70=0.008838394187856443, P71=0.006730105006830536, P72=0.0076144533725123285, P73=0.007693805455666713, P74=0.0070266152205036205, P75=0.005205855195030673, P76=0.007092512240132692, P77=0.013035645106899094, P78=0.010663503937376811, P79=0.011870999799544637, P80=0.013417995709028904, P81=0.016671397906951213, P82=0.028594283132593053, P83=0.027733946178882807, P84=0.023340181522879297, P85=0.022011013965259014, P86=0.008406789909420589, P87=0.007085275458561499, P88=0.019185644392029605, P89=0.016142393257885745, P90=0.010329912855989007, P91=0.015263478639704217, P92=0.018541002132166085, P93=0.009697361911118953, P94=0.012781820811431536, P95=0.01621641637744533, P96=0.008875648968082952, P97=0.007485306400800073, P98=0.00796906571806959, P99=0.005468568302292141, P100=0.0025704988022138267, P101=0.0037247480981696572, P102=0.0024147806861566897, P103=0.0011915519728688212, P104=0.0015299129673114105, P105=0.0009698554940657588, P106=0.0011694700511963208, P107=0.000875720221349997, P108=0.0009996381876090981, P109=0.0009575975609150842, P110=0.0020911733040398298, P111=0.002746552020254588, P112=0.0007335233618098425, P113=0.0012566997241585305, P114=0.001499861104692432, P115=0.0017004566189504828, P116=0.0018188969943627703, P117=0.0018839327223144464, P118=0.0016863256300960715, P119=0.0009291132733012693, P120=0.0009854228907581707, P121=0.0009575618303304153, P122=0.001013281911927732, P123=0.0006048707474258961, P124=0.000838645841370636, P125=0.00138125910711867, P126=0.0008256313221280054, P127=0.000400144036632778, P128=0.0004847768940954043, P129=0.00018474223065945839, P130=0.00014887488272930286, P131=8.939126873882062e-05, P132=0.0001274042225378258, P133=0.00038165732844084156, P134=0.0002794357513960187, P135=0.00010145889755678126, P136=0.0002464452990356151, P137=0.0001840017947287597, P138=0.00029107818296034046, P139=0.000350418640792981, P140=0.00016978086786234025, P141=7.055844517307794e-05, P142=3.1874066579852234e-05, P143=2.129532582559445e-05, P144=2.175856838423674e-05, P145=2.4419600227137684e-05, P146=3.6103810594080814e-05, P147=5.5043438363347515e-05, P148=9.680150787519204e-05, P149=0.00016757509698912018, P150=0.00015894685225198, P151=0.00011552890611745567, P152=7.25556446198087e-05, P153=7.478674598226973e-05, P154=2.97618608387945e-05, P155=1.4048886245465078e-05, P156=8.88942986614388e-06, P157=1.1531113488514578e-05, P158=1.5487272766511156e-05, P159=3.1876487679930894e-05, P160=2.1304711536647193e-05, P161=2.2920496700551712e-05, P162=2.7411364655105752e-05, P163=3.1610732194957406e-05, P164=5.081062891834913e-05, P165=0.00010333009358360608, P166=0.00010210078213790903, P167=5.799581716063615e-05, P168=5.209273354474807e-05, P169=6.162015915253961e-05, P170=7.622842382096335e-05, P171=0.00011156436653837027, P172=0.00011610197165142081, P173=0.00015244556214502687, P174=0.00017748939922952657, P175=0.00016876690532248, P176=0.00011559567945309496, P177=0.00014694858546407673, P178=8.850308091144614e-05, P179=1.3201843574779223e-05),Row( P0=-685.7423521559253, P1=65.12558374252873, P2=-12.247154176945745, P3=1.7378405605099005, P4=-10.763044680420002, P5=-11.419092442541988, P6=-9.469431055349832, P7=-7.818789731542355, P8=-2.0896264266945908, P9=-8.062339813643394, P10=-6.103461336691106, P11=-4.414789500041557, P12=-2.373423603341621, P13=1.0961073576831657, P14=-2.010723604115303, P15=10.21908172999759, P16=6.344691727352011, P17=4.167758617324277, P18=10.72813492541467, P19=7.110276109372166, P20=3.1883796680641714, P21=5.5129491143386025, P22=5.018841407732798, P23=0.6602486979462199, P24=-2.3061205479659663, P25=-2.326786461416484, P26=-2.9492016333038484, P27=-0.24483742806929293, P28=0.02924698564606063, P29=0.3604501962647026, P30=-2.1254071942181287, P31=-0.9774041736159763, P32=-1.6112410159020585, P33=-1.1132412221856698, P34=-3.5024432333314466, P35=-0.20086195089165632, P36=-5.091980460862315, P37=-1.3864916201665742, P38=-1.3267125256437664, P39=2.3337985034877216, P40=0.6534139158590598, P41=0.62195048877951, P42=0.6492958509803083, P43=0.642696961819532, P44=0.6231272271330143, P45=0.6302137646255076, P46=0.6460263773814742, P47=0.6693606035421199, P48=0.6976329384881956, P49=0.6475941549483961, P50=0.6322899439641445, P51=0.638283618169098, P52=4.874446327131138e-05, P53=3.506939319639042e-05, P54=5.0582533558291615e-05, P55=1.1479692573844194e-05, P56=1.1763617296399988e-05, P57=1.5243543142852047e-05, P58=8.264824861924042e-06, P59=5.121612629590205e-06, P60=2.044173802231322e-05, P61=0.00034179360469986376, P62=0.0054876141190034105, P63=0.01958211735827384, P64=0.041600112016309865, P65=0.023492213093935687, P66=0.014874064015898233, P67=0.019881624686865187, P68=0.019919163356764136, P69=0.021903598742158907, P70=0.02711329569290778, P71=0.015709137162767502, P72=0.0014284754981019744, P73=0.0007776879953986035, P74=0.001279873856771755, P75=0.0014779586772648502, P76=0.004408706343930887, P77=0.012035846656445017, P78=0.006452873796291702, P79=0.003999815090677966, P80=0.002222078699781879, P81=0.0015855503247035002, P82=0.0018968501303876762, P83=0.002498497996404908, P84=0.005862167282257817, P85=0.009951644157244085, P86=0.003920849926211993, P87=0.00512743507963444, P88=0.006674301067509646, P89=0.009767757987464707, P90=0.005669672273616577, P91=0.0013322512558388514, P92=0.0010222176185491888, P93=0.0006127179605616843, P94=0.0008082123330170765, P95=0.0006832862093188241, P96=0.0007067085639430255, P97=0.002032919406106229, P98=0.001822883159119051, P99=0.00040961743154618917, P100=0.00023165958763192456, P101=0.00028044150657919334, P102=0.00026767707113983975, P103=0.00034548998554352216, P104=0.0005299320372105395, P105=0.0005270892248767924, P106=0.0006312582387555144, P107=0.00043738414109568014, P108=0.0007557940323159074, P109=0.0012292588943609204, P110=0.0014544994073035847, P111=0.001085993028548436, P112=0.0004098324757050949, P113=0.0006286733606857305, P114=0.0006831630172050944, P115=0.0002585070482376795, P116=0.0001271388916062831, P117=0.0006273554904184526, P118=0.0005125119746378316, P119=0.00027425619384790833, P120=8.612954513467623e-05, P121=7.565660049687431e-05, P122=0.00011354322588225, P123=0.0002809906738466833, P124=6.360938206176262e-05, P125=3.589171399328122e-05, P126=3.317470016944336e-05, P127=1.322763767278108e-05, P128=1.5118917130013067e-05, P129=8.177691061549371e-06, P130=1.128142353126806e-05, P131=2.0269802966420655e-05, P132=3.4756709029328214e-05, P133=3.108511111607094e-05, P134=3.3636411427852495e-05, P135=5.0340580481266423e-05, P136=6.0996304487310464e-05, P137=5.410624212269898e-05, P138=5.700562788187713e-05, P139=7.712231881496808e-05, P140=2.892914923382811e-05, P141=1.2714954124282512e-05, P142=1.712615365068692e-05, P143=1.4644110853650292e-05, P144=1.796794278703791e-05, P145=9.665044824532337e-06, P146=1.4583631114603035e-05, P147=3.0217134415383943e-05, P148=1.5462371943095317e-05, P149=3.616290830841127e-05, P150=1.4401038186641974e-05, P151=2.358800724474884e-05, P152=9.32968065989823e-06, P153=1.6785475414932674e-05, P154=9.548218435350177e-06, P155=9.406775487124483e-06, P156=9.11398157682902e-06, P157=7.967135682835581e-06, P158=6.396520111589134e-06, P159=6.208958340276183e-06, P160=6.348091933795658e-06, P161=7.965248424758802e-06, P162=5.841693527347806e-06, P163=5.181618025139097e-06, P164=4.787704585065615e-06, P165=3.3604392374453004e-06, P166=3.6501808918566074e-06, P167=3.9602133693725915e-06, P168=6.3831667397110315e-06, P169=4.457543388385986e-06, P170=3.3684843352619852e-06, P171=2.9724982667070262e-06, P172=7.590387446965557e-06, P173=5.638261241188863e-06, P174=4.03880416340972e-06, P175=4.498562153681181e-06, P176=3.905890558976317e-06, P177=6.233420587261836e-06, P178=5.511650457798521e-06, P179=7.578267416155811e-07),Row( P0=-696.1134172086219, P1=49.803498715671246, P2=-13.783036744565678, P3=-0.31393804842909123, P4=-13.949769413388664, P5=-13.624335623584049, P6=-10.806617739200307, P7=-11.127711297601483, P8=-7.308368940168658, P9=-8.6294614964613, P10=-11.003933195965502, P11=-9.25365861580545, P12=-8.345414036842925, P13=-2.746156877376604, P14=-4.652934747147506, P15=6.726540641299006, P16=4.888446646279752, P17=6.4279633153782445, P18=13.24546504045858, P19=8.258480231807914, P20=1.9464102652578152, P21=2.9323171958475953, P22=2.6443804179170844, P23=1.3075247857959722, P24=-0.5103867731252769, P25=-2.026346668401326, P26=-4.600794141252857, P27=-1.3659537687577226, P28=-4.06669285700522, P29=-1.388593834978041, P30=-2.8714171822221557, P31=0.17435994521405404, P32=-1.8236800295219282, P33=0.41516011105775147, P34=-2.721343610471032, P35=0.7280745821438155, P36=-0.8488188357420309, P37=1.1528766739836127, P38=-0.7882930011768828, P39=0.1482670835655668, P40=0.6399309819686338, P41=0.5905439096371989, P42=0.5432304878221385, P43=0.5438211640854291, P44=0.5745731395084014, P45=0.6029001055541943, P46=0.6371571719013408, P47=0.6465121034428424, P48=0.6213519303469625, P49=0.6558257200637616, P50=0.6635498584716637, P51=0.6728694720929128, P52=1.4055729108838437e-05, P53=1.9402109574229827e-06, P54=1.0451658740232306e-06, P55=3.7224248254170243e-06, P56=8.328957920842067e-06, P57=8.498783267960185e-06, P58=3.426646900839854e-06, P59=2.541602747658766e-06, P60=1.0230139251841058e-05, P61=0.0005316987280644512, P62=0.007277275654467683, P63=0.024797994058934014, P64=0.018412435480981305, P65=0.027659437855897133, P66=0.04881268509101698, P67=0.05321730905140128, P68=0.03893995528700821, P69=0.03466916487088944, P70=0.009366887034429311, P71=0.0006969259627229153, P72=0.00024012553239862984, P73=0.0006201854208263601, P74=0.0008965302019826811, P75=0.0016585045538689584, P76=0.0012742320069299513, P77=0.0015442453796407766, P78=0.002365201616149373, P79=0.0043019893160149356, P80=0.005872709290843572, P81=0.008037022177962495, P82=0.013153628741553527, P83=0.017446338228614123, P84=0.015413092459983341, P85=0.01420761228988626, P86=0.009425121124827808, P87=0.005126601095384776, P88=0.000994926582240775, P89=0.0003234012883497045, P90=0.0004054482661472513, P91=0.0011551194004746944, P92=0.0021995149984632935, P93=0.0010004175763809536, P94=0.0008869427331642716, P95=0.000599950016646048, P96=0.00027629588405016843, P97=0.000428943312116523, P98=0.0002818823981181243, P99=0.0001235523840396988, P100=0.00036402022541189954, P101=0.0005184739949897792, P102=0.00042936060023948637, P103=0.0003340849408924002, P104=0.00012274970544237166, P105=0.00010391806497900066, P106=0.0002355488225061511, P107=0.00040589351569142587, P108=0.0005171828534366999, P109=0.00031254043139521056, P110=0.000600453824883938, P111=0.00035171745885977324, P112=0.00011280148997835394, P113=0.00014730365047084081, P114=0.0001294023272414959, P115=7.64407443347376e-05, P116=0.00010355756426368524, P117=0.00011596359875390688, P118=0.00012647609231790886, P119=0.00010448972351481744, P120=7.854419247578317e-05, P121=9.128990912606782e-05, P122=0.0001505592679543051, P123=4.291902834768966e-05, P124=4.180331110031007e-05, P125=5.272905545590096e-05, P126=3.0474922203080075e-05, P127=2.5480040921694635e-05, P128=3.150636016333915e-05, P129=6.432642465411689e-05, P130=2.6828445149670307e-05, P131=2.786487084989941e-05, P132=3.864363416413701e-05, P133=4.1209130552644844e-05, P134=3.897401863940659e-05, P135=4.8163602042427914e-05, P136=8.275706898845145e-05, P137=6.450289676193634e-05, P138=5.2508823132155494e-05, P139=4.962497494564595e-05, P140=2.4619944960300505e-05, P141=1.787408402585779e-05, P142=1.3749503490491219e-05, P143=8.500434400507246e-06, P144=1.0348002737792045e-05, P145=6.673831859954843e-06, P146=1.1406901993391023e-05, P147=1.1368432140616384e-05, P148=2.979574770084595e-05, P149=2.1115625705081284e-05, P150=1.551106722199436e-05, P151=1.626857792022464e-05, P152=1.1200969861778163e-05, P153=1.0832211878448008e-05, P154=1.701453332462843e-05, P155=1.2575919912723955e-05, P156=9.810320470731613e-06, P157=1.0065470724640564e-05, P158=1.2150059938335281e-05, P159=9.435639665307395e-06, P160=6.110306585297693e-06, P161=3.810263136229652e-06, P162=4.861915958151331e-06, P163=5.298110173098157e-06, P164=4.5439272841409125e-06, P165=5.3501818859845744e-06, P166=5.041888690033579e-06, P167=7.307688702455533e-06, P168=5.7820352948420086e-06, P169=4.9878941005020825e-06, P170=3.4096594648962727e-06, P171=3.695120458511674e-06, P172=3.838292858068248e-06, P173=4.398628149794676e-06, P174=5.0104264105695875e-06, P175=4.2629554781111216e-06, P176=3.405092352873845e-06, P177=5.342493982430485e-06, P178=4.662710349107487e-06, P179=3.9231243115768876e-07),Row( P0=-666.7849448973677, P1=47.76426577641035, P2=-20.334835016373837, P3=-6.810404568667796, P4=-15.08997418773435, P5=-15.631507212820901, P6=-11.559833323235711, P7=-6.537851998772431, P8=-6.360075374767708, P9=-17.230570423371734, P10=-10.134654267287697, P11=-5.279707270761271, P12=-10.703550818310989, P13=-4.455567701914737, P14=-6.292775580305945, P15=1.0735837093234437, P16=-0.29512473777707415, P17=-0.6814752723597803, P18=5.7763986875297535, P19=0.5749928570833902, P20=2.161767820122955, P21=6.38069180825077, P22=4.215283618312209, P23=5.341074940854092, P24=7.7175008121472075, P25=5.643455639033268, P26=3.2848452508884076, P27=2.846036051086547, P28=-1.4673019262586722, P29=0.15150454537160987, P30=-3.204749918505273, P31=0.5961171452133274, P32=-2.484445056746914, P33=-2.2176565158327572, P34=-4.348828393007327, P35=-2.8698216027166445, P36=-2.2020234823238556, P37=-1.9446102655248267, P38=-2.2019265823255925, P39=-1.3628727138663643, P40=0.6238819625353881, P41=0.6339160669683173, P42=0.6226974668373869, P43=0.6367081659545459, P44=0.6658742116833162, P45=0.7195034403945334, P46=0.7098909261097806, P47=0.6786697585010776, P48=0.6271359334788474, P49=0.6324753212209355, P50=0.660161888854224, P51=0.661367285932904, P52=1.9060267003375597e-06, P53=1.615579534064268e-06, P54=4.389013065195145e-06, P55=1.924766901247088e-06, P56=9.329212306501562e-07, P57=1.4844661270285026e-06, P58=8.924565783034871e-07, P59=1.7281453589639992e-05, P60=0.001202576566993261, P61=0.011508792874033976, P62=0.04163054079705908, P63=0.059245682955711254, P64=0.026914357302069015, P65=0.03346628001755426, P66=0.02998899733721405, P67=0.009874850044916256, P68=0.004748710703199284, P69=0.00924497218411637, P70=0.008614224708183918, P71=0.004111516116321904, P72=0.006553289246395426, P73=0.008870839743526818, P74=0.01408357199940756, P75=0.012946697852449344, P76=0.009025400710280055, P77=0.008168831265525603, P78=0.015766256412497615, P79=0.03243037012012288, P80=0.02239254832824412, P81=0.013542582787638943, P82=0.011397433711763886, P83=0.010580074763723699, P84=0.012516711313013665, P85=0.017527880451375797, P86=0.024549543541614833, P87=0.011209529638475895, P88=0.009661206929846983, P89=0.006368916683975663, P90=0.012998316093509871, P91=0.043301690253385504, P92=0.061916369091199214, P93=0.0293164940324172, P94=0.030405606174849387, P95=0.015917800913262656, P96=0.0019044043393091497, P97=0.0024737255656530578, P98=0.00324380344148707, P99=0.01152474313773562, P100=0.01060772299014926, P101=0.007594977690852991, P102=0.005503652737546741, P103=0.0021401173098746875, P104=0.0008484559370950095, P105=0.0004362489691115852, P106=0.0014490195551133708, P107=0.0017433096491148165, P108=0.0011218406818244893, P109=0.0011237737020218864, P110=0.0030992409614463232, P111=0.0035539302832412866, P112=0.0027882320475246876, P113=0.002690425635404991, P114=0.003611025115648868, P115=0.0038278258104467466, P116=0.00500546669922774, P117=0.002137479296327709, P118=0.0010527513218912276, P119=0.000519221778118621, P120=0.0013161921235735907, P121=0.001576814565265381, P122=0.0008795049521431492, P123=0.001209863033005957, P124=0.0008247421602964452, P125=0.0007311532873152766, P126=0.0008434360587269068, P127=0.00030923985735093477, P128=0.00032476134274281765, P129=9.90341430286492e-05, P130=7.431123034720312e-05, P131=0.0001158534165168179, P132=4.7100219196092226e-05, P133=0.00011889988343704008, P134=0.00011086028712490964, P135=0.00018489921698048006, P136=0.00030667342183360313, P137=0.0002389507661036005, P138=0.00018806849881118057, P139=0.00013691964861720933, P140=6.660917847164718e-05, P141=5.1704161192224016e-05, P142=3.5045957936503206e-05, P143=3.5253335078671837e-05, P144=5.66352349433551e-05, P145=8.033508718247567e-05, P146=0.0001473281215453992, P147=0.00013801698738234348, P148=0.00012369304383071987, P149=0.00014704550327385353, P150=8.847637248292044e-05, P151=7.619278168422513e-05, P152=5.579126977347583e-05, P153=3.6708326596757785e-05, P154=1.5889884299883593e-05, P155=1.2359198868221747e-05, P156=1.3284810112585864e-05, P157=2.6217307521982658e-05, P158=2.669464883772494e-05, P159=3.182075088065526e-05, P160=2.6709206985510862e-05, P161=3.3031058488809e-05, P162=3.0100301479379513e-05, P163=2.6217057483251513e-05, P164=4.085835566783797e-05, P165=6.674492521352008e-05, P166=7.17060853173689e-05, P167=4.677301532556214e-05, P168=3.2119599973687806e-05, P169=2.112274470819649e-05, P170=2.6681993877828352e-05, P171=4.5340088336115904e-05, P172=6.510825525723531e-05, P173=7.288453340585146e-05, P174=8.576286198294121e-05, P175=8.926291610364562e-05, P176=0.00011347450457772925, P177=8.893677782261733e-05, P178=4.0040385082574534e-05, P179=4.980781326475824e-06),Row( P0=-678.1926415699761, P1=54.36309458278151, P2=-11.931239259122918, P3=-0.8065522456222813, P4=-10.590632586513557, P5=-11.158389306398634, P6=-7.30198393237287, P7=-9.409571351632211, P8=-3.7389674833217788, P9=-7.41664771530212, P10=-8.564491006353133, P11=-4.8417311496193385, P12=-5.147753722235556, P13=1.9220256157355977, P14=0.2661993486767254, P15=9.621329454841485, P16=7.469125169244151, P17=5.115952874088627, P18=12.019622713041876, P19=8.597417194709598, P20=5.439887885879087, P21=6.979969504591575, P22=4.312864232090936, P23=1.2970500091254364, P24=-1.2774882421570495, P25=-2.2947905911623465, P26=-0.958400682918442, P27=2.050611669725692, P28=-0.4178494835696757, P29=0.18100256215420488, P30=-2.8769944224477504, P31=-0.5237087290424707, P32=-2.1455963216547596, P33=-0.7267355396376766, P34=-4.853799285093905, P35=-1.5481007185165072, P36=-4.829214926013005, P37=-0.4678356844244691, P38=-1.1140680572245163, P39=1.297508901979857, P40=0.6480262605606324, P41=0.6287070713221409, P42=0.616818908543859, P43=0.6231720604198216, P44=0.5893444849144507, P45=0.5980674444685065, P46=0.6054317584925675, P47=0.6102040838142111, P48=0.6521036039131854, P49=0.6488210232691801, P50=0.6194150902834266, P51=0.6200280629600349, P52=0.000162684167521854, P53=2.417704073920902e-05, P54=2.4747347084865706e-05, P55=1.2927199327415622e-05, P56=1.0030214791159052e-05, P57=9.851777671702992e-06, P58=2.341342514391082e-06, P59=2.8062404590072263e-06, P60=2.9008926285207734e-05, P61=0.0008823681792536183, P62=0.006490222860243326, P63=0.015128947712238565, P64=0.04757709202871046, P65=0.0453580759856012, P66=0.026137016034257544, P67=0.03239616657928911, P68=0.03557206663076892, P69=0.04379860387373244, P70=0.04437618218148814, P71=0.017381726051396713, P72=0.001135931435855114, P73=0.0010757554063549905, P74=0.0009989098021550777, P75=0.0009264572024714567, P76=0.001795543608951576, P77=0.009097068948940693, P78=0.012925602588681908, P79=0.011138450004037797, P80=0.006104250766823445, P81=0.0022508617310126274, P82=0.0020014191802071022, P83=0.002817048768446522, P84=0.009096375999142207, P85=0.016331551831368137, P86=0.008427282538418096, P87=0.012339509604827189, P88=0.01084589782074231, P89=0.007915292622388876, P90=0.004317123430551667, P91=0.002734388834413654, P92=0.0024477202286917834, P93=0.001042230532216168, P94=0.0010303653347180375, P95=0.0007956447286177076, P96=0.0005262256881337202, P97=0.0012771429119116289, P98=0.002291026085077453, P99=0.0013690452699080219, P100=0.0006790169908585445, P101=0.0006203633496375008, P102=0.0004232212727961083, P103=0.000381545729144383, P104=0.0005119765945474987, P105=0.0015881250322412038, P106=0.002230285381048591, P107=0.0007632838469404479, P108=0.0007062204266070439, P109=0.0007326839073875336, P110=0.001101497828880068, P111=0.0013716694718805256, P112=0.0007811970531855929, P113=0.0009256511527166378, P114=0.0008667577778545796, P115=0.00046459327925213234, P116=0.0003974280406127021, P117=0.000651729842112723, P118=0.0002981116360561357, P119=0.00019355598756079444, P120=0.0004348865475235837, P121=0.00014440819809322472, P122=0.0001248400944036212, P123=0.0001313649572259353, P124=6.334504629157296e-05, P125=5.5570304547680116e-05, P126=6.465649986999919e-05, P127=3.099634693110923e-05, P128=1.517913259358074e-05, P129=7.17430329172037e-05, P130=0.00014282583619030996, P131=8.258052833969899e-05, P132=6.394871732965477e-05, P133=0.00010904210520070813, P134=8.478792670105059e-05, P135=7.320445547182958e-05, P136=0.00012108650637091272, P137=0.00012433720128019498, P138=6.33513337181383e-05, P139=9.784502963903651e-05, P140=6.180621525799974e-05, P141=4.75180687064286e-05, P142=6.64747793421394e-05, P143=5.671622890871995e-05, P144=6.461474051937024e-05, P145=4.135301098963138e-05, P146=2.649410262514024e-05, P147=3.1568086903306254e-05, P148=3.669918033295728e-05, P149=4.445724197923815e-05, P150=2.464118385500004e-05, P151=2.226759018102714e-05, P152=1.4681322703996987e-05, P153=1.5723784856626517e-05, P154=1.4797790153748296e-05, P155=9.366463892258045e-06, P156=8.946346744260665e-06, P157=1.2390373370114909e-05, P158=1.1814204643694542e-05, P159=1.2008894295052604e-05, P160=9.91944411874019e-06, P161=7.647092358619013e-06, P162=7.618740086627392e-06, P163=1.0312085353246039e-05, P164=9.245111208343262e-06, P165=8.102814273742419e-06, P166=5.496746057877112e-06, P167=6.455981440474821e-06, P168=7.201751220705672e-06, P169=6.281161995225891e-06, P170=4.524052983482685e-06, P171=8.784214159388146e-06, P172=9.925518567565274e-06, P173=5.498180724167215e-06, P174=2.6073767051667814e-05, P175=1.390021617152802e-05, P176=1.283871667361533e-05, P177=2.1124933073737405e-05, P178=9.414143463774433e-06, P179=1.2050728465309182e-06),Row( P0=-557.7683499054824, P1=29.86348141143536, P2=-25.83714859826351, P3=-10.403580952973497, P4=-16.171679380161027, P5=-17.70938864469651, P6=-11.101097083139752, P7=-10.053042544056654, P8=-4.13836314458138, P9=-10.698458832530305, P10=-9.485553287773042, P11=3.0660542256182346, P12=-5.089757783193041, P13=7.935298046409612, P14=0.5256025435948467, P15=10.305997197389798, P16=7.012133300049294, P17=4.992283133920052, P18=12.369737141710251, P19=6.577667115507823, P20=5.154227801758009, P21=3.311167477874079, P22=-4.592513711807983, P23=-1.1077868119121896, P24=2.148982749977151, P25=3.7024133140227002, P26=0.03288435192286153, P27=-0.8425544567009241, P28=-3.30712938808654, P29=-0.46657853308239605, P30=-0.3962395912064068, P31=3.046156072144972, P32=-2.3222138894137774, P33=0.14572760867889234, P34=-3.750888228313478, P35=2.59947717652972, P36=0.06719936632546075, P37=1.0784555551918096, P38=-1.200586989505987, P39=-1.290731891822702, P40=0.6450878621896845, P41=0.6075623304008682, P42=0.6027920199570168, P43=0.6271900950758599, P44=0.623551087610879, P45=0.6627352114660825, P46=0.6792457572361675, P47=0.5967665200310811, P48=0.5388534600816418, P49=0.54799355322336, P50=0.5992106243311236, P51=0.6699568657091719, P52=0.0004787338566767192, P53=3.958406305107096e-05, P54=1.2294129563942836e-05, P55=1.4193169961367094e-05, P56=2.5330269511435877e-05, P57=1.6816858775030376e-05, P58=3.50243357402395e-06, P59=6.804471731597968e-06, P60=0.00017614602629455428, P61=0.002133218214479624, P62=0.01480481244202112, P63=0.029262749251065204, P64=0.04097471935622327, P65=0.18302951769168105, P66=0.3353533387883447, P67=0.23553563086146176, P68=0.07531199913592757, P69=0.0849712124633557, P70=0.11447876457583506, P71=0.12354589918171446, P72=0.17187556635293666, P73=0.231078712285445, P74=0.26538241420591246, P75=0.049631501624182076, P76=0.008443337974239224, P77=0.013759633026060897, P78=0.044430716215822716, P79=0.21283952980367268, P80=0.28905551044822453, P81=0.25061923070846903, P82=0.2975333481691133, P83=0.13439648266007592, P84=0.06190830762630739, P85=0.05631322793488012, P86=0.048160362504063345, P87=0.12789375338485995, P88=0.22475397719253154, P89=0.13127850067115954, P90=0.13287405688863904, P91=0.2564527582199782, P92=0.4205577708232321, P93=0.2731480584257558, P94=0.207378522235631, P95=0.16698270550614444, P96=0.014395288173135058, P97=0.0035668214227940515, P98=0.003984888391698312, P99=0.008429229621688655, P100=0.06668612974669351, P101=0.13957550758418363, P102=0.11859942286806785, P103=0.07529516653554609, P104=0.06356793146369658, P105=0.034598340952261664, P106=0.013903002772939755, P107=0.003930994385052675, P108=0.01617084223641201, P109=0.027466415557075408, P110=0.10581184818260843, P111=0.13376851443479648, P112=0.02300594557468622, P113=0.036204089896626476, P114=0.06728298160748872, P115=0.05408503475383544, P116=0.04745518904315152, P117=0.022796947522733927, P118=0.03263613690557045, P119=0.022526279601428743, P120=0.013164493165267264, P121=0.009247809563713348, P122=0.014433882535723143, P123=0.01774310658333047, P124=0.00814152464417025, P125=0.008132299981451529, P126=0.003924359028464195, P127=0.010478492619513769, P128=0.0029432718843418804, P129=0.0008744181685583129, P130=0.0025957117098942798, P131=0.0029151788379485183, P132=0.003791501773203897, P133=0.0022730136080014057, P134=0.0010321819629012143, P135=0.001763311733822517, P136=0.007263991847853775, P137=0.005929667993094076, P138=0.005279744717854465, P139=0.0025684955281395773, P140=0.0019821020030782433, P141=0.0009192093441256501, P142=0.0005039007648162842, P143=0.0011198554492416386, P144=0.0007809336206820939, P145=0.0006971156574318246, P146=0.0011634376045796318, P147=0.0011162822759145756, P148=0.001457085088670647, P149=0.001402818501842203, P150=0.0014660009558032016, P151=0.0016448668573621273, P152=0.0009306857890915731, P153=0.0006287713390248582, P154=0.0002855895738470297, P155=9.6035825142404e-05, P156=5.035147053848558e-05, P157=7.78775834833093e-05, P158=8.312254436596117e-05, P159=0.00013120197346849573, P160=0.00019358841510689468, P161=0.00024391477044117036, P162=0.00018683273603289447, P163=0.0002011159436132852, P164=0.0002790635688920514, P165=0.0003639952316232362, P166=0.0006968483468917626, P167=0.0006998116795026563, P168=0.0006809771402653934, P169=0.0007120958360777206, P170=0.0007594609748816052, P171=0.0007280172052945698, P172=0.001005334061685401, P173=0.0013158954991644711, P174=0.0018823664204765567, P175=0.002039326695560447, P176=0.0012427459134866552, P177=0.0010686104249540557, P178=0.0005917953869461461, P179=4.967717609353006e-05),Row( P0=-692.4385991757839, P1=63.02824150862052, P2=-5.164821341652339, P3=4.040510413400536, P4=-10.24482454145503, P5=-11.370924490692023, P6=-6.299195919985024, P7=-8.458453414578994, P8=-2.9907274301631808, P9=-6.0226892948131505, P10=-9.291078279112165, P11=-4.782879164473806, P12=-6.5684397305633535, P13=-1.959900051627087, P14=-2.5128418383995075, P15=7.6140524219773, P16=5.598348366059374, P17=4.253266159106515, P18=9.959679607295515, P19=8.55296650101587, P20=2.6270566051562945, P21=3.613393781434284, P22=1.9404035367725891, P23=-0.38635489667443107, P24=-0.36132380042430845, P25=-1.6281930281728572, P26=-3.5070940843531786, P27=-0.9435582742362877, P28=-1.5848723297424048, P29=-0.7720973666008684, P30=-1.730976711574186, P31=0.3331339249055872, P32=-0.5097667514358272, P33=-0.12081106569463465, P34=-4.484313701000388, P35=-1.4017992323633275, P36=-4.002572761440577, P37=-0.1813510853574892, P38=-1.0634236458350061, P39=-0.2975336591553724, P40=0.6569028683220975, P41=0.6070040354110714, P42=0.5960843912543556, P43=0.6098003770253165, P44=0.6109959498470021, P45=0.6295908966016771, P46=0.643810178967809, P47=0.610643922107402, P48=0.6323240238804839, P49=0.6560989560212289, P50=0.6407048302676753, P51=0.6704380131972135, P52=5.652316550571811e-05, P53=1.2880315399481924e-05, P54=1.4373855570313905e-05, P55=1.8734461764501653e-05, P56=3.903316983400297e-05, P57=3.337580646940525e-05, P58=1.493911426319183e-05, P59=8.460111843111074e-06, P60=8.803515310972118e-05, P61=0.0017089264121549273, P62=0.02313124580472199, P63=0.03736066080620008, P64=0.01916840715265421, P65=0.031801228245686784, P66=0.0447697255192537, P67=0.057236933390312696, P68=0.036355754727545556, P69=0.03782064068587025, P70=0.013431153350458776, P71=0.0008824663977744862, P72=8.714359571139573e-05, P73=0.00028505902047730247, P74=0.0009503626285466239, P75=0.000874802697716258, P76=0.0012003502303326037, P77=0.0038248436939420792, P78=0.006867404201503739, P79=0.008254884228943195, P80=0.007827882745581114, P81=0.007335468132134625, P82=0.008347757075950753, P83=0.016247736404038815, P84=0.008942952777379965, P85=0.006178961567852567, P86=0.006996892734052763, P87=0.006555988129144985, P88=0.0019825116855464115, P89=0.0008732513536165472, P90=0.0024182619866330497, P91=0.0037185876323418657, P92=0.0017616395901261935, P93=0.000342488043304991, P94=0.0003475644633348441, P95=0.00040605338127824186, P96=0.0001946084397682616, P97=0.00035448039207253065, P98=0.0011219730995695133, P99=0.0010614091748262441, P100=0.00048368697902414094, P101=0.00023737613659440473, P102=0.000140908073163219, P103=0.0002040233986673469, P104=0.0004384493174732625, P105=0.0006120332740322415, P106=0.0005879249107453678, P107=0.00018044083619925537, P108=0.0001335116097735518, P109=8.44247427997135e-05, P110=0.0005384021198209494, P111=0.0009700970497489824, P112=0.0003090986062348445, P113=0.00010111402994718623, P114=0.0001296084290143301, P115=0.000298170417704825, P116=0.00039198838069784723, P117=0.0001545607742404428, P118=6.039312105628988e-05, P119=0.0001293287554283499, P120=0.0003044242392937317, P121=0.0002782510503627588, P122=4.447972032414025e-05, P123=4.553291740802898e-05, P124=9.333377097732775e-05, P125=0.00011688500591851008, P126=3.096843220692893e-05, P127=3.31228382956946e-05, P128=3.2082093502188694e-05, P129=4.175315831179187e-05, P130=1.3463796392942984e-05, P131=1.594997006724137e-05, P132=2.511717942710999e-05, P133=3.816893133048549e-05, P134=2.437743777279922e-05, P135=4.194543995946862e-05, P136=5.691553418846993e-05, P137=3.157770693909922e-05, P138=9.943412544363093e-05, P139=8.81241459339718e-05, P140=2.178025133297758e-05, P141=2.1510059386806086e-05, P142=1.3217169565165419e-05, P143=1.526638737286432e-05, P144=1.4130241520408282e-05, P145=2.1491598681567033e-05, P146=2.3171750153056706e-05, P147=2.130303272769589e-05, P148=3.07544791014266e-05, P149=3.102710986707014e-05, P150=1.722771595948354e-05, P151=1.3730850864841997e-05, P152=7.5721557917068926e-06, P153=7.071351508996481e-06, P154=6.56811247835015e-06, P155=3.3936329743071807e-06, P156=3.4462845707741585e-06, P157=4.5553045035426425e-06, P158=4.461668445834421e-06, P159=4.993065410663048e-06, P160=4.256070520469659e-06, P161=5.57112802842593e-06, P162=4.663085000418902e-06, P163=4.694058184762952e-06, P164=3.884226847772705e-06, P165=3.446794256907804e-06, P166=6.390571385959458e-06, P167=7.127266311658762e-06, P168=6.235352239966805e-06, P169=4.4154792856581264e-06, P170=5.3220393285040985e-06, P171=5.523980813626269e-06, P172=4.648083192341298e-06, P173=6.0638431440091455e-06, P174=1.2260736453651591e-05, P175=1.1207807649252323e-05, P176=7.999103385053459e-06, P177=9.037761597050848e-06, P178=5.156734670851266e-06, P179=5.972055774868245e-07),Row( P0=-623.1481963819871, P1=52.31971123573954, P2=-25.15345491544323, P3=2.9097307320904484, P4=-11.755656969778766, P5=-11.394534394716638, P6=-1.9346017089167262, P7=-6.425190408041281, P8=0.04629263324520986, P9=-6.723876344383373, P10=-6.6705886949725475, P11=3.09070293010313, P12=0.21843977023887512, P13=6.717838730684635, P14=1.598816053895679, P15=10.279537302420314, P16=7.746055927467256, P17=5.644990945814825, P18=12.013966434648296, P19=7.91257057519265, P20=4.93077757975598, P21=7.601290039424184, P22=2.3720108450046955, P23=4.01477270357303, P24=2.98179111162378, P25=2.561771774526891, P26=-0.326274849445805, P27=3.3434778437556947, P28=-0.11507622186156595, P29=1.8301023902026918, P30=-1.9546628946978175, P31=0.6262546946814336, P32=-1.4379280815619566, P33=1.9433002466619411, P34=0.23869975802009988, P35=0.9452106438698339, P36=-3.2105573051730123, P37=-1.2138791163942895, P38=1.0693435111400953, P39=2.5862537166736894, P40=0.6028553635046832, P41=0.606250340111912, P42=0.5933548473083707, P43=0.6306267413054673, P44=0.6740234316432792, P45=0.7178876995635473, P46=0.6501729894581594, P47=0.680398635287647, P48=0.6575786682060875, P49=0.5999454419590707, P50=0.5639565880580462, P51=0.6134280967796151, P52=0.03749538990884896, P53=0.00375546618467447, P54=0.00013476712298185652, P55=4.2000079452002836e-05, P56=1.641181885151646e-05, P57=1.4889973635593794e-05, P58=7.92247290157743e-06, P59=5.805913068196372e-06, P60=0.0002663892018437769, P61=0.0034908917907557784, P62=0.010566960938062815, P63=0.038148503170176984, P64=0.06423011153197476, P65=0.06276629392450662, P66=0.08238853205091204, P67=0.09500985142523907, P68=0.04983177218527126, P69=0.02828157374699318, P70=0.026289941129538422, P71=0.043834359512921035, P72=0.08222647838108783, P73=0.041121946552386844, P74=0.006572352155919346, P75=0.00805430552584569, P76=0.01698957345944645, P77=0.04301041494927881, P78=0.0492501512572346, P79=0.029553745417095618, P80=0.020383081142231748, P81=0.03652568421422569, P82=0.06723398020510572, P83=0.08415668306669363, P84=0.05264989887710056, P85=0.017108570783268464, P86=0.010154207473928895, P87=0.026972435444476424, P88=0.042314911000956856, P89=0.0219529587032529, P90=0.02815212594732412, P91=0.05191541235379602, P92=0.05959384114741228, P93=0.02073548634945186, P94=0.03516987422583625, P95=0.06708095318642537, P96=0.07565209866844592, P97=0.048849197926546205, P98=0.04872671263863126, P99=0.016167719151621504, P100=0.007470209887150657, P101=0.007154872125037922, P102=0.006195457068674609, P103=0.016658019204109926, P104=0.008049704815680622, P105=0.002829724202815373, P106=0.015651159986639043, P107=0.019632759767423808, P108=0.015184031874738229, P109=0.012996556021297493, P110=0.02509169576395548, P111=0.017290750963876254, P112=0.002279095033719843, P113=0.013066135627073615, P114=0.018945675010233975, P115=0.007779228868730185, P116=0.004984866308894759, P117=0.006001331164513779, P118=0.012810042181061423, P119=0.005455702806669339, P120=0.00438222140503721, P121=0.005365825225422895, P122=0.0044407550544786935, P123=0.0013673410101095154, P124=0.004304492552836266, P125=0.005828249866948113, P126=0.001337868573181517, P127=0.0014359030610904704, P128=0.0009471196371048872, P129=0.0004660828492165193, P130=0.0011456866748200372, P131=0.0003282774973862901, P132=0.0004484560353139744, P133=0.0004351282228482343, P134=0.0010235145409337738, P135=0.0008493895754255918, P136=0.0008136345348229662, P137=0.0023581521678080904, P138=0.0028107600473104358, P139=0.0007430194580918756, P140=0.000443480078739576, P141=0.0003277639981000728, P142=0.00020126462050862274, P143=0.0001399041110315984, P144=0.0002527343863350466, P145=0.00019050386164066633, P146=0.0002387585639226007, P147=0.00040198032204808224, P148=0.0003586347556004154, P149=0.000377729487253633, P150=0.00020199538618574469, P151=0.00014621225406402576, P152=8.935293786182884e-05, P153=8.034824597718625e-05, P154=5.822864820128417e-05, P155=6.743695991957445e-05, P156=3.7401478305669644e-05, P157=4.474175672036436e-05, P158=4.704022332694983e-05, P159=4.7638926060636446e-05, P160=3.568422011453497e-05, P161=4.032783870497296e-05, P162=3.57740010618782e-05, P163=1.6522860853555238e-05, P164=2.3169807464071627e-05, P165=2.524950862731919e-05, P166=1.9391252646113893e-05, P167=2.2954754044453123e-05, P168=2.5438040477220865e-05, P169=2.2328039411269133e-05, P170=2.013869748211898e-05, P171=1.3343950634129913e-05, P172=1.847680493721248e-05, P173=1.2234954342086605e-05, P174=3.642346988884338e-05, P175=2.4065752336880735e-05, P176=1.5158624603058573e-05, P177=1.4394094870577546e-05, P178=1.172932849510562e-05, P179=1.5007638356297551e-06),Row( P0=-542.1238603920825, P1=36.87472127847164, P2=-38.74861558218379, P3=-12.467400925322748, P4=-20.470261259669073, P5=-16.113528262117345, P6=-14.294738083688529, P7=-11.360786426447573, P8=-2.4097330764368396, P9=-11.821977631032611, P10=-1.8514970741810386, P11=11.568373195901563, P12=0.07736659958372076, P13=5.782475842269799, P14=-3.6440610504808655, P15=1.3585413039526106, P16=1.196010033713132, P17=5.402479846468573, P18=9.146312466590626, P19=4.800295392154946, P20=4.811166136190924, P21=4.823978416464124, P22=3.0701801458657165, P23=4.744667728130058, P24=-1.5920491481786696, P25=-1.9223209842797233, P26=1.1068443819528497, P27=2.3535078075096476, P28=-0.4386694643208805, P29=2.1334227261254575, P30=-1.3214249914047806, P31=1.8704110593196557, P32=-0.8393269908554756, P33=-0.4716040360809938, P34=0.6659372694867696, P35=2.7577939606900133, P36=0.3121686276312858, P37=2.3623963951481834, P38=0.5364022902509253, P39=1.6389270062523555, P40=0.5155865646119172, P41=0.5306258277202768, P42=0.5504402935668524, P43=0.5784377871934115, P44=0.5392957948040924, P45=0.5764837059012563, P46=0.6288118732912494, P47=0.6602859749655964, P48=0.6484434444459259, P49=0.6402225973196812, P50=0.6097734736725668, P51=0.5818368315507876, P52=0.001854010821734268, P53=0.00017345381442468512, P54=2.3507508551607043e-06, P55=1.6676111063708795e-06, P56=1.0279094076422421e-05, P57=3.759315383432674e-05, P58=2.1330848315027576e-05, P59=1.7475451905491517e-05, P60=3.9791050441951206e-05, P61=0.00016737856754848557, P62=0.0029922713066928225, P63=0.008004315580903438, P64=0.011584892971849935, P65=0.05088534549629408, P66=0.0907250440724677, P67=0.06297771481100457, P68=0.04686422068532639, P69=0.10037814308487862, P70=0.19549152467790173, P71=0.1382176517107058, P72=0.09638709093976057, P73=0.08812322261494702, P74=0.11347643219597293, P75=0.14297321470924695, P76=0.18203198647373123, P77=0.3342824369185079, P78=0.44674717678867804, P79=0.6135721705568989, P80=0.3013466510633854, P81=0.13928786055942574, P82=0.08005312471984005, P83=0.022354683103497638, P84=0.02024663203886149, P85=0.02454152172652937, P86=0.026094695912921826, P87=0.04047430089616769, P88=0.07078212809027618, P89=0.05552673493463782, P90=0.06320679781643741, P91=0.08190657181178623, P92=0.09099406293570496, P93=0.07030016013244123, P94=0.12272919230846871, P95=0.15544302085338685, P96=0.1420883980680168, P97=0.16753770584909602, P98=0.1836262245636924, P99=0.2934917772140635, P100=0.1250363292659983, P101=0.07308274502932766, P102=0.04910274383703262, P103=0.02649303688628623, P104=0.014509783428272843, P105=0.009515304603398155, P106=0.011537161269623815, P107=0.012601978309757001, P108=0.01821739634658042, P109=0.023764166126878185, P110=0.06608586505844088, P111=0.0716103646093915, P112=0.016974718225669934, P113=0.00857351440022538, P114=0.011921438576070692, P115=0.01193718814327475, P116=0.03044965114457228, P117=0.03896552561270365, P118=0.03396553690037795, P119=0.02398730279731585, P120=0.026744733148212418, P121=0.01565320141122133, P122=0.011447253163062995, P123=0.00887117680490288, P124=0.01940075954701223, P125=0.0180276362427272, P126=0.005688116587085457, P127=0.009903881039074674, P128=0.011593212102660313, P129=0.0022065101070029443, P130=0.0006103738006939109, P131=0.0013745990084060907, P132=0.0012399375600714144, P133=0.0024721301590888284, P134=0.001689262488004034, P135=0.0004211117196413063, P136=0.000969562192853427, P137=0.0015759270347596073, P138=0.0028341244815066, P139=0.0023090551817697403, P140=0.0013603254138983638, P141=0.0010473000438265324, P142=0.0008444102411805738, P143=0.0007139931376519915, P144=0.0004093147614404688, P145=0.0004495285565433516, P146=0.0006497826592735406, P147=0.0007666298859774521, P148=0.0007115331097257701, P149=0.0017312737894918165, P150=0.0010400395205640263, P151=0.0006623565986900692, P152=0.0006205205617772571, P153=0.00025198131386333124, P154=0.00020993260177146547, P155=0.0002427877326693135, P156=0.0002442686707753363, P157=0.0001983972217853985, P158=0.00018671069670250245, P159=0.0001518384057638637, P160=0.00011192359839097011, P161=0.00014936824446850722, P162=0.00015838552384655024, P163=0.0001742953567404386, P164=0.00018651286900424582, P165=0.00025645936223783695, P166=0.00016729593273426523, P167=0.00016640494198261714, P168=0.00015957068853928724, P169=0.00016786050976211388, P170=0.00013169965853411685, P171=8.695357044770293e-05, P172=7.727160011006837e-05, P173=6.577177098436361e-05, P174=7.504075540387523e-05, P175=8.788512064517971e-05, P176=8.82580967760049e-05, P177=5.541843862727642e-05, P178=2.3392090298874822e-05, P179=2.632046995739194e-06),Row( P0=-674.009072456776, P1=48.562987495933314, P2=-10.408026316102076, P3=-2.3719128399529175, P4=-16.701285897964862, P5=-17.981549931482807, P6=-3.801496121131051, P7=1.0574306138838623, P8=-3.1621630910129626, P9=-4.087107159820163, P10=-5.886457858233721, P11=-3.941223125579891, P12=-3.589552362755832, P13=-2.5182456764891197, P14=-1.2918864865130182, P15=-2.4043178273600616, P16=3.444595116410644, P17=0.6799360313472325, P18=3.573293294107587, P19=9.228766037935811, P20=6.816266465799117, P21=16.86117499354719, P22=14.632722774804815, P23=12.403611490897164, P24=3.645040963166215, P25=5.6806085370815405, P26=0.45437169877186956, P27=0.8781880839819943, P28=1.821038461685215, P29=0.605882958232647, P30=-0.8063483806793414, P31=-0.7333896684835328, P32=-1.7466810733072797, P33=-1.4088868905788334, P34=-2.7989167478798405, P35=-3.2533180764049536, P36=-2.864310062224258, P37=1.0364423558576616, P38=-0.9089731442476578, P39=1.5827517773355504, P40=0.5948698354708151, P41=0.6372127329528848, P42=0.5925137950355259, P43=0.6130503971618948, P44=0.6309699310276474, P45=0.6617283524726404, P46=0.6911421909346238, P47=0.6585256231552379, P48=0.6436840321383899, P49=0.5930369903550132, P50=0.5212277923708507, P51=0.5197802690594484, P52=5.119302429987963e-05, P53=2.355457814668872e-05, P54=1.671026133355639e-05, P55=5.886047831032435e-06, P56=2.075369062623229e-06, P57=2.5769520385708464e-06, P58=6.976899607592151e-06, P59=4.8631592776691064e-05, P60=0.0005093917968984523, P61=0.0042847062924418, P62=0.03827959524993584, P63=0.06994850221953709, P64=0.040435358535460375, P65=0.024120817042606556, P66=0.004246877698461525, P67=0.0012224250970944315, P68=0.0005942149541190851, P69=0.000512179052795342, P70=0.0006662760059623374, P71=0.0011911626547405744, P72=0.0049803302976990215, P73=0.01386522069110977, P74=0.028287711049975413, P75=0.02892626911815395, P76=0.02135257554943383, P77=0.057040860612686504, P78=0.09630102640259372, P79=0.05207695335827676, P80=0.01120666643423068, P81=0.0035784928783497666, P82=0.0033838917314847664, P83=0.005084038512496051, P84=0.012919905214360317, P85=0.05896171935088996, P86=0.06758148519982655, P87=0.03233132311008923, P88=0.021134888189465423, P89=0.01890899217842401, P90=0.06954043899432003, P91=0.0498351537415578, P92=0.006570860984516319, P93=0.0009121651852691666, P94=0.0011651923947708074, P95=0.0011147798516062584, P96=0.003230065629647603, P97=0.005264569167511327, P98=0.01563066781206464, P99=0.032702239489115804, P100=0.005215690947691186, P101=0.0003475680864321676, P102=0.00039150079048695945, P103=0.00031702338261879444, P104=0.00010729533639697574, P105=0.0002691677447312328, P106=0.001554520736540904, P107=0.0013660903628025618, P108=0.00023254580326542154, P109=0.00010920041242500724, P110=4.4922317966391296e-05, P111=0.0010327230260064075, P112=0.0012974266185915252, P113=0.0006333211611046344, P114=0.000277930197306708, P115=8.512583887202314e-05, P116=0.00010864766668373273, P117=0.00022161288377079298, P118=0.0002682641008397179, P119=0.0001001951201120442, P120=0.00015529986930154545, P121=0.00040488481122287164, P122=0.000245250287289582, P123=0.00015495305864244354, P124=0.00018884819166252752, P125=0.00022479108827766167, P126=0.00013012720304180522, P127=0.0001063934092254899, P128=0.0001844524001851576, P129=9.930305362549658e-05, P130=0.00022346829851857384, P131=0.0002650101436811801, P132=0.00017429500251694486, P133=0.00030925026434550925, P134=0.0002533308010698087, P135=0.00027972748582635095, P136=0.00012016077187299738, P137=6.435597299910889e-05, P138=2.4056454825157146e-05, P139=2.1563269420195345e-05, P140=3.569915569306809e-05, P141=1.764292431250485e-05, P142=3.9328476103274244e-05, P143=4.767630730473023e-05, P144=3.997146122819856e-05, P145=3.089491001914887e-05, P146=2.0713483269578807e-05, P147=2.13852757231473e-05, P148=2.775664174458711e-05, P149=4.5682994094037455e-05, P150=4.5561167517456707e-05, P151=4.925732645120823e-05, P152=9.536221215140719e-05, P153=9.369490326690244e-05, P154=6.49966046530686e-05, P155=8.547016745022889e-05, P156=0.00011581955521744828, P157=0.00013397811937573606, P158=6.582368499629078e-05, P159=4.1854802441100515e-05, P160=9.880105873969523e-05, P161=0.00018725258512198488, P162=0.00016556877574710932, P163=0.0001579426567041924, P164=0.00017876996563975008, P165=8.381021963238838e-05, P166=6.6191218188401e-05, P167=4.8069725596577034e-05, P168=3.0959549445708996e-05, P169=4.393616250435725e-05, P170=6.35420583831586e-05, P171=6.603025877970314e-05, P172=5.351269577242353e-05, P173=4.544582756283213e-05, P174=9.186195658148944e-05, P175=0.0001337285676983092, P176=0.00016370493111650026, P177=0.000120618418438849, P178=5.374734114433695e-05, P179=7.289678736507825e-06),Row( P0=-669.9846852056414, P1=55.82069345413922, P2=-11.185528268743393, P3=1.6010545780694174, P4=-14.963111200585981, P5=-15.791183010228336, P6=-1.8784250157800622, P7=2.632535887412153, P8=4.056724296553348, P9=-4.946063401635517, P10=-6.591501574194804, P11=-0.9224765109950578, P12=-5.272808667248506, P13=-1.638068467485676, P14=4.695352088094277, P15=1.111790884047177, P16=4.942474816630194, P17=8.288480474260552, P18=7.109674119280738, P19=14.583342372774673, P20=11.558694133949464, P21=16.555562529596468, P22=17.027165122333216, P23=10.417015660164605, P24=5.11694439279545, P25=6.292401136844213, P26=3.0115428250796095, P27=3.95474845062526, P28=2.6409288657747454, P29=4.7841213719890705, P30=-1.1492758179811047, P31=3.008698883775579, P32=-2.1311686007138015, P33=2.8031799098448484, P34=-1.7050521768261628, P35=-1.6961453296048858, P36=-0.022031145390509977, P37=1.281285666688069, P38=-0.8787962746614204, P39=1.2648061651726592, P40=0.6068848933703075, P41=0.6202219814324281, P42=0.6157969876134802, P43=0.5645947891630223, P44=0.5962706489678297, P45=0.6301331758643924, P46=0.6467992395737233, P47=0.6513681127570852, P48=0.6450042804869575, P49=0.6556299035793164, P50=0.6190455287912112, P51=0.5858472470137411, P52=0.0008923114482386285, P53=3.425553943168298e-05, P54=1.404344311430231e-05, P55=7.746276898768854e-06, P56=4.280571298241359e-06, P57=4.693439612270314e-06, P58=2.98341443657806e-06, P59=7.067933738971806e-06, P60=0.0001534824074236234, P61=0.0015548064646845915, P62=0.018272865728683092, P63=0.04613584834756383, P64=0.04371329792801738, P65=0.0373094368527055, P66=0.03866158395365363, P67=0.02496203846502489, P68=0.004192241684837856, P69=0.0003979629193913887, P70=0.00041175748945273844, P71=0.00024827049778168613, P72=0.0003157303471081044, P73=0.0017564191028146506, P74=0.005529865507676928, P75=0.011087984433217386, P76=0.022123788149712092, P77=0.027292111491010313, P78=0.03162154855379853, P79=0.034466966791769145, P80=0.020415225812621163, P81=0.020291804953076135, P82=0.030602517369002876, P83=0.007366346419640135, P84=0.002349314594255068, P85=0.00909276563986739, P86=0.013551951641678448, P87=0.03235653414504377, P88=0.0737851800587574, P89=0.027625328163004947, P90=0.009455550160576863, P91=0.0055510361303672305, P92=0.002411026798853438, P93=0.0006741815422706804, P94=0.0009386491554759368, P95=0.0007027881481155229, P96=0.0030786126203359047, P97=0.015783540315923904, P98=0.013999370889413594, P99=0.01026591609065604, P100=0.0014884315656930928, P101=0.00020106965325451744, P102=8.851322567075453e-05, P103=0.00037218771490054263, P104=0.0009667195107168673, P105=0.0006478516724090588, P106=0.0007347689121966004, P107=0.00032916277355254285, P108=0.0002134067682249926, P109=0.000494408205336212, P110=0.000773844329435248, P111=0.0007725735509091666, P112=0.0007357192215546486, P113=0.000453648722380022, P114=0.0003050530238374308, P115=0.0004546394638139451, P116=0.00020696506048119105, P117=8.190684098631495e-05, P118=0.0004835488264466727, P119=0.0009544478212728983, P120=0.0004833219933141736, P121=0.00019756340770909394, P122=0.0008534807617736285, P123=0.001404097736404289, P124=0.00030447874302677504, P125=0.00026688639512683005, P126=0.00015909946451882033, P127=0.00020611789818740595, P128=0.0009216983965021866, P129=0.00045166952015277526, P130=7.70413421012295e-05, P131=0.00030178989498511104, P132=0.00025919987758745, P133=0.00012133677310988006, P134=0.0003038508858074152, P135=0.0002535966598964818, P136=0.00010627810374899719, P137=0.00017475546304116467, P138=5.670697039207767e-05, P139=3.84296760736217e-05, P140=5.028861955807521e-05, P141=5.52858188308245e-05, P142=7.36333843026926e-05, P143=0.00013059779509538808, P144=0.00019994451538559685, P145=0.0001257710339775358, P146=5.584641751106297e-05, P147=4.2495756355833466e-05, P148=3.644578785377096e-05, P149=4.820665646581411e-05, P150=3.0477969397915046e-05, P151=2.1646085710149907e-05, P152=1.4704461652839092e-05, P153=9.500789214367848e-06, P154=7.455262148845308e-06, P155=8.277107040014317e-06, P156=1.5924029664638037e-05, P157=1.8721926363425397e-05, P158=3.0313454225192364e-05, P159=3.2127364431891646e-05, P160=2.500011204002179e-05, P161=2.1458137550676375e-05, P162=1.264915596897399e-05, P163=4.81191130097192e-06, P164=4.794247371122812e-06, P165=5.6386659669519205e-06, P166=7.688002512683182e-06, P167=1.0345574463344329e-05, P168=1.6010951512719216e-05, P169=1.3038092506295292e-05, P170=5.204763693736543e-06, P171=6.635055190820582e-06, P172=7.896770818330745e-06, P173=8.74719845297763e-06, P174=1.2747740469034258e-05, P175=1.009309107474296e-05, P176=1.693590292942947e-05, P177=1.664755600282923e-05, P178=6.2236605452255815e-06, P179=5.557747076413759e-07),Row( P0=-505.5974066087696, P1=23.06223008906836, P2=-30.97243082483297, P3=-10.727769636950262, P4=-23.736022535779032, P5=-24.62662212921742, P6=-5.916039041622585, P7=-5.522753934773612, P8=-1.5358232284352384, P9=0.41218227199463886, P10=2.658199729186964, P11=7.272009447209179, P12=8.965903019733656, P13=13.576640673990584, P14=10.056452616540156, P15=5.311446007454691, P16=3.5328820067241122, P17=1.6263222456003954, P18=4.617240413351991, P19=7.4704652102794595, P20=3.2757455728355906, P21=6.1629738486330705, P22=-0.16797868848301925, P23=6.147855662984884, P24=5.514920579431459, P25=8.386619745298116, P26=1.5129228693050258, P27=-2.1184823892271547, P28=1.3455914727017366, P29=4.480487399552091, P30=0.34268229238511627, P31=2.985122772350437, P32=-0.13779372456661607, P33=4.1751354974103565, P34=1.2859905078159737, P35=-0.3855564877974599, P36=1.8487045917064893, P37=5.5241291608841, P38=0.0630774067429913, P39=1.2541074488464055, P40=0.5839002934897085, P41=0.5907803371747907, P42=0.5616357151957434, P43=0.5721462886396931, P44=0.6651953746210858, P45=0.6453305725334484, P46=0.5371420234384734, P47=0.48414887905386955, P48=0.5105650384785867, P49=0.5521229519471407, P50=0.6028761456191472, P51=0.5994546034100295, P52=0.020006035938549643, P53=0.001447107898405195, P54=9.871561226127916e-05, P55=8.115854995589494e-06, P56=5.592233563114718e-06, P57=8.217644738532312e-06, P58=6.57641827910777e-06, P59=7.450209216941304e-06, P60=1.568428260138843e-05, P61=5.482914154504371e-05, P62=0.0005553440499523339, P63=0.002234442952145185, P64=0.002679552497531975, P65=0.00509837768790683, P66=0.016244922488134663, P67=0.03488857703581484, P68=0.03674711200355637, P69=0.04845818551894357, P70=0.1068295719687839, P71=0.15753982755555773, P72=0.3865033900130298, P73=0.6953135866242601, P74=0.6992589089229051, P75=0.5221207637430784, P76=0.1970342396489706, P77=0.17040626024410477, P78=0.3940425706313714, P79=0.5080479279340948, P80=0.42970386107310715, P81=0.24786733072179623, P82=0.5907495971439792, P83=0.8135605040516316, P84=0.5657209785183217, P85=0.3089784125055625, P86=0.35275347702265275, P87=0.48909443766677435, P88=0.9107361547947403, P89=0.7585697441150804, P90=0.7966212129261612, P91=1.258376297726367, P92=1.2785223732333217, P93=0.6740942633157979, P94=0.6730467867771747, P95=0.6211812634237452, P96=0.19077756112414446, P97=0.03041119879621179, P98=0.029542453916772436, P99=0.01851152722390923, P100=0.014351683026079805, P101=0.014989533033207216, P102=0.08332592439905649, P103=0.0951984691059007, P104=0.06071245634057263, P105=0.03217192298131359, P106=0.09647312648097733, P107=0.09529023002819463, P108=0.03422210904127344, P109=0.0075309464731822305, P110=0.006601061599957006, P111=0.04285858214763227, P112=0.10391750494499466, P113=0.1589234344823347, P114=0.05035888417695974, P115=0.01969838017688284, P116=0.00971067309689934, P117=0.0036974358704609934, P118=0.006560111996339361, P119=0.0334219402544225, P120=0.036391518073965186, P121=0.008210255585304306, P122=0.017418211543817185, P123=0.019940810250200797, P124=0.023329726744505765, P125=0.03491487037867851, P126=0.026683268523001104, P127=0.024321945835236693, P128=0.016799075932794068, P129=0.034923780886011876, P130=0.024664489111536035, P131=0.029480093332422606, P132=0.019630885831779944, P133=0.02837374121477547, P134=0.047268654888531804, P135=0.05870878521774223, P136=0.03463287431679214, P137=0.01684360324640751, P138=0.012733953045189138, P139=0.008897362116731427, P140=0.0036807743992264778, P141=0.00510550677742011, P142=0.005320282312472979, P143=0.005528943480320785, P144=0.0016850762067655545, P145=0.0016227161269137004, P146=0.0012110988567322057, P147=0.002107606622836224, P148=0.0016222377971819265, P149=0.0034387085339415554, P150=0.006610439112088556, P151=0.0045966139237994955, P152=0.0017087357949859962, P153=0.0007683197497129434, P154=0.00036332642923051167, P155=0.00022431224582692685, P156=0.00024136693082154668, P157=0.00018469785102135438, P158=0.00020678591541033615, P159=0.0002827777236934838, P160=0.0002726901809467273, P161=0.00030042480160972373, P162=0.00032542018285833577, P163=0.0003929455421799923, P164=0.0002851673294833964, P165=0.0002514522776314141, P166=0.0002195001524921514, P167=0.00024406332234481649, P168=0.00028744910817583825, P169=0.00039899538693832483, P170=0.00036106530125131853, P171=0.00020959616346615702, P172=0.00017212133198978886, P173=0.0003046732938530044, P174=0.0006029009483482726, P175=0.0007237104617905354, P176=0.0008416432504966649, P177=0.0008016988384528032, P178=0.00030823381979095543, P179=2.7192381205328324e-05),Row( P0=-598.6811846101557, P1=66.40912879320106, P2=7.568949312042457, P3=1.9413814696170608, P4=-16.514594139383522, P5=-9.829570846412206, P6=3.8954060543048916, P7=-1.4282304383569664, P8=-3.8766744006525364, P9=1.3915960257225288, P10=-2.958585150350291, P11=-7.49752004797253, P12=-1.813956193323898, P13=-6.349260796446148, P14=-3.9535706816278338, P15=0.6978673669590116, P16=-1.7495800435552475, P17=2.3694850250450816, P18=-0.978935972727559, P19=3.229442536946548, P20=-1.7672876309658692, P21=1.6378259765064356, P22=-0.5766413291107348, P23=0.4453199168737008, P24=-0.9470294458561466, P25=3.4350131965361546, P26=-0.5130364281801044, P27=2.5748674481921356, P28=1.8271791132715312, P29=1.4182638927687545, P30=0.33909601026990127, P31=4.056882489301385, P32=-0.5319721618155013, P33=4.422417009255258, P34=-1.132874995561388, P35=2.1448218127855654, P36=1.74426379364781, P37=3.1306016423397343, P38=0.7008720618612811, P39=0.6205023806144747, P40=0.6897921328361035, P41=0.6354650847340978, P42=0.6710269830228693, P43=0.6524391408619639, P44=0.7076565008598463, P45=0.7491186339590488, P46=0.6978188003684241, P47=0.6946958949179974, P48=0.6905607385804882, P49=0.7000390777324325, P50=0.7051376234553797, P51=0.7180230519887103, P52=8.187323629455496e-05, P53=0.00011582675498690085, P54=0.00011417233825896476, P55=0.00014992014987709186, P56=0.0008182999433322842, P57=0.0033675312512944735, P58=0.007403343344556393, P59=0.011791367981768195, P60=0.011586240187627213, P61=0.013639464613702038, P62=0.02096675637240025, P63=0.029221492119026377, P64=0.022531802137771277, P65=0.026968877996975484, P66=0.028582761951152315, P67=0.012999894876280482, P68=0.007834432630012872, P69=0.009184333939036001, P70=0.015502616619153007, P71=0.011029205133664993, P72=0.006530015719653634, P73=0.010470186104719186, P74=0.010584150130138803, P75=0.010705338990451303, P76=0.014175626057821371, P77=0.01973242161107417, P78=0.024545833002743624, P79=0.03606610614037693, P80=0.032350179483119995, P81=0.024514050003228798, P82=0.02754510079164848, P83=0.02968370611872323, P84=0.031571434332181944, P85=0.0288549017971837, P86=0.03215044642586853, P87=0.048441572606195235, P88=0.07148903933386268, P89=0.04072411826226813, P90=0.023299274701651603, P91=0.03624703545833373, P92=0.05475643358276467, P93=0.026890599210827645, P94=0.017604417442622894, P95=0.03429978183067968, P96=0.08967547185098411, P97=0.053904174177928255, P98=0.03938357891253536, P99=0.02596187761881303, P100=0.010663247772505336, P101=0.028294857572059764, P102=0.024443103619323003, P103=0.010081246242943865, P104=0.008103656488676947, P105=0.004013442613486362, P106=0.008381129435728307, P107=0.007446255886824786, P108=0.00681426206208738, P109=0.006863879566800299, P110=0.003998552045213975, P111=0.0006686159547639087, P112=0.0005319160098854009, P113=0.0016583116218034989, P114=0.0015340957745058788, P115=0.0006142136570811526, P116=0.0007466160570304007, P117=0.0002727402351281706, P118=0.00031813987149264926, P119=0.000382510396128927, P120=0.00028871719398716614, P121=0.00020087609392217937, P122=0.00014375611915801333, P123=0.00015297805466519006, P124=0.0001915638485160467, P125=0.0002073261996218125, P126=0.00024143065159222334, P127=0.00043986439890295097, P128=0.00045026322182861717, P129=0.0005027645519019519, P130=0.0003775342954619485, P131=0.0008463552407194419, P132=0.0007696687939906632, P133=0.001301138595060466, P134=0.0021784322603646306, P135=0.003494170806358034, P136=0.005018541744761119, P137=0.004109952298398365, P138=0.0023276764698615384, P139=0.0016969347188233938, P140=0.0018245764602855402, P141=0.0022923055037359186, P142=0.00295778697454286, P143=0.006682529809644776, P144=0.0037724343230567964, P145=0.0022357305916830116, P146=0.0006783650638214379, P147=0.00031140720527989414, P148=0.0001679938683575946, P149=5.9747380245401866e-05, P150=5.095513526961363e-05, P151=0.0001028885920691479, P152=0.00012909768563989756, P153=0.00011873307633404205, P154=0.00024131851287878374, P155=0.00027191830558412745, P156=0.00021865058443574863, P157=0.0002758691155779977, P158=0.0003503665119606287, P159=0.00029135350685028597, P160=0.00029022563847188465, P161=0.00019432326534999934, P162=0.00011608453117920436, P163=0.00010457079354951542, P164=0.00015420726305779482, P165=0.00020444336677289968, P166=0.00028541314497020233, P167=0.0003770366738788946, P168=0.0003696004628481583, P169=0.0002851202796307833, P170=0.00016232536556960023, P171=0.00014305244148747411, P172=0.00016952112988993716, P173=0.00027349891281617405, P174=0.00026428830962270194, P175=0.0002810428483497625, P176=0.00022655533370116534, P177=0.000147488536989043, P178=7.449107435099977e-05, P179=1.077202952126421e-05),Row( P0=-478.23486424969576, P1=42.4469994756548, P2=-22.14478137146913, P3=-0.5964892422119236, P4=-12.722463963542936, P5=-18.649824244077184, P6=1.2457919563463944, P7=5.03589988420607, P8=-7.965912836001239, P9=-2.507040195539789, P10=-7.78050343422806, P11=-0.19593912894776272, P12=-1.0161132210175803, P13=7.942004810657452, P14=17.30684562518947, P15=15.872612950683276, P16=20.175758272436777, P17=11.50766668609423, P18=5.045919757597034, P19=6.152075202898011, P20=-4.9061643546414295, P21=6.297181666470154, P22=4.130234703838998, P23=3.3040185964935316, P24=-2.2175468923689645, P25=-0.599227691619559, P26=-0.2826645869062663, P27=4.387115219690128, P28=7.479255874140217, P29=2.838734776960751, P30=0.7250104011154834, P31=1.819779126099587, P32=1.1307750512445003, P33=3.6975041321408058, P34=-2.260204472945649, P35=2.102791183546203, P36=0.4454867070706939, P37=3.277334599606451, P38=-0.25293318197298936, P39=1.482331500224624, P40=0.6923711860141151, P41=0.7416785499466076, P42=0.7787266232134499, P43=0.639150850924328, P44=0.558037896706845, P45=0.5272263718208537, P46=0.5466228899609001, P47=0.5471651497785934, P48=0.5339716197318505, P49=0.5696285575005872, P50=0.5994028356334798, P51=0.65206913269582, P52=0.018259795236577996, P53=0.0024690672231255616, P54=0.0006330531244592313, P55=0.0002874518795398383, P56=0.0004281093140863442, P57=0.0022917017943014104, P58=0.005243325056138885, P59=0.0039965638689644515, P60=0.009781493770229027, P61=0.01693872278693589, P62=0.015758137490265212, P63=0.02334918968526027, P64=0.03648017515153061, P65=0.09967979965285162, P66=0.22230789310877552, P67=0.3884901643429315, P68=0.8794828980237697, P69=1.9983766166699675, P70=1.633388329991125, P71=0.43695882940766384, P72=0.05087128697917868, P73=0.024770243731866447, P74=0.010229715705066474, P75=0.005903530573133826, P76=0.007156084406666967, P77=0.0160922552850156, P78=0.02669849811115091, P79=0.04040487646076466, P80=0.06107811372760398, P81=0.1260026219476866, P82=0.23374858359853104, P83=0.5531919092748334, P84=0.8900042253048696, P85=0.963477953245423, P86=1.7861215040867762, P87=1.9680277301777085, P88=1.8405707938221378, P89=0.46393645347959656, P90=0.09691611482533821, P91=0.10984791652649867, P92=0.12545506380440474, P93=0.28317569932273445, P94=0.7631296837953595, P95=0.5277267682451209, P96=0.6110952460288507, P97=0.5957229131388232, P98=0.5352003544422578, P99=0.5927912443629134, P100=0.3725247706585956, P101=0.23593955730752256, P102=0.08415841176417628, P103=0.07962884521026517, P104=0.06626714371597345, P105=0.14486005245149172, P106=0.10150801798422067, P107=0.05119400376272623, P108=0.04329068917557542, P109=0.05985343333593277, P110=0.021326909618991474, P111=0.0344715833774455, P112=0.06005244081026711, P113=0.08539749512306587, P114=0.018185241050986938, P115=0.004649487394313486, P116=0.006856558727187628, P117=0.033656493103665044, P118=0.03241660516848874, P119=0.006946445480799479, P120=0.01440478555338135, P121=0.02554173466086403, P122=0.05153690145189458, P123=0.019343308515879405, P124=0.020172779327917792, P125=0.023971078905865802, P126=0.06498679712081139, P127=0.05508862325732342, P128=0.06721183685602967, P129=0.041586415657898924, P130=0.046443375957113155, P131=0.018403918568142663, P132=0.015177757405123607, P133=0.042995407977972876, P134=0.02307370775890194, P135=0.018191094770510098, P136=0.024009617787848842, P137=0.02183516576364129, P138=0.008371250447486225, P139=0.0030293952093226493, P140=0.002256699544620396, P141=0.001773035888313661, P142=0.0025121874563061833, P143=0.004114819795554432, P144=0.0076704418632441065, P145=0.00861184057582804, P146=0.004722079727343618, P147=0.004001564214930648, P148=0.002195060236782566, P149=0.0039960706547985305, P150=0.002081542257163841, P151=0.00225831510652479, P152=0.0004882991217088834, P153=0.0002570443781455685, P154=0.00021989740219863937, P155=0.0002778978515642045, P156=0.0004632059102929657, P157=0.0005577503753841896, P158=0.0010790509395816278, P159=0.0016664654108517521, P160=0.002248345374566607, P161=0.0032088278845553396, P162=0.002002643296274928, P163=0.001153893743849388, P164=0.0009703001660364887, P165=0.0009265934389735485, P166=0.0013088386376174984, P167=0.002342759800976629, P168=0.001779079275056901, P169=0.001346419498380964, P170=0.0007821721020246377, P171=0.0003728110549606506, P172=0.00025608083715535106, P173=0.00039022222444119725, P174=0.0006288081080905443, P175=0.000501228569894094, P176=0.00043983100450151744, P177=0.0003276381430704992, P178=0.00013928522385799668, P179=1.2501007702164432e-05),Row( P0=-701.4746115455052, P1=81.06434627686154, P2=-9.735822065038347, P3=5.351893731723122, P4=-6.692671984613175, P5=-13.314011839430343, P6=-4.576953202561479, P7=1.1165921871212214, P8=1.0431941488606906, P9=-8.415534982066808, P10=-1.9008942699494376, P11=-3.8491110471303442, P12=-7.39789636365027, P13=-1.813225560969781, P14=-2.5511058516777285, P15=-5.962484406915602, P16=2.455471666288562, P17=-2.372478332059333, P18=-4.357093885127761, P19=0.864080637943023, P20=-7.085244809203985, P21=0.05272589801687681, P22=-2.6608623251992523, P23=-4.476362204036756, P24=-4.067874535542083, P25=0.6310959717472958, P26=-2.166857073440729, P27=3.32383827043868, P28=3.380709745826099, P29=6.991564691137987, P30=2.510450342012189, P31=10.674986342113057, P32=3.4510547430699434, P33=5.30740467242342, P34=-2.205591072133881, P35=0.39483182635660036, P36=1.186124261090748, P37=4.834132335725001, P38=-0.15733586237969224, P39=-0.15031610582980584, P40=0.7543823689437361, P41=0.7946090931807052, P42=0.7648381402409229, P43=0.6824354723211192, P44=0.6812839143696073, P45=0.7024332589334801, P46=0.6744423880864711, P47=0.6462328806587604, P48=0.6744752505265802, P49=0.7294876093816768, P50=0.739475561771478, P51=0.7289747919975276, P52=5.278133031249099e-06, P53=2.2227574448248073e-05, P54=1.96360366577624e-05, P55=2.0290004927112574e-06, P56=1.966174898858446e-05, P57=0.00034622949201227227, P58=0.0013489200181843674, P59=0.004669716965265987, P60=0.007688133712284685, P61=0.003775549837007954, P62=0.0008787711040080089, P63=0.00013853590035189714, P64=0.00014023481174313773, P65=0.0004315379039773561, P66=0.0014390122502027966, P67=0.003690828488013186, P68=0.007658766587160346, P69=0.0076821619130721394, P70=0.0054834981402028865, P71=0.0032724970127253734, P72=0.00047997790406520234, P73=0.00039970980868480124, P74=0.000567766510373349, P75=0.0010269145674797657, P76=0.0019627614961351828, P77=0.007918037706076322, P78=0.00906009700818995, P79=0.007231415232269283, P80=0.006423422210009868, P81=0.004556520850876409, P82=0.002080832456862625, P83=0.0023206881470126325, P84=0.0034124276168517012, P85=0.007333480139512604, P86=0.007629131594093581, P87=0.004616018045187916, P88=0.0054353176266602665, P89=0.004627376205759803, P90=0.0029763464717751363, P91=0.0034555117062555494, P92=0.006637300530003276, P93=0.006539393967789607, P94=0.008267497966397124, P95=0.0035892282733936457, P96=0.0033122389945101087, P97=0.0016861638173728017, P98=0.0008799879350476511, P99=0.0038157493389489256, P100=0.003412461618913095, P101=0.0028939949254581016, P102=0.00106672985314736, P103=0.0001398533662682075, P104=0.00011272050904662788, P105=9.340843864468072e-05, P106=0.0002014750040543504, P107=0.00038779585140650424, P108=0.0009447502308421626, P109=0.00068348652888924, P110=0.0001992657864149625, P111=0.0003314897117474448, P112=0.0007207725428689354, P113=0.00045799001218826475, P114=0.00046106101266510217, P115=0.0004194667132667658, P116=0.00019296900561279658, P117=0.0001385776978350584, P118=0.000217599243767595, P119=0.0002668988587228185, P120=0.00012039267980605736, P121=0.00018069527505806482, P122=0.0003192907417707243, P123=0.00013428733666039062, P124=0.0006044115648818779, P125=0.0002691099065432345, P126=0.00014594283225288307, P127=8.029698089511563e-05, P128=0.00012009367640099675, P129=8.192689827060018e-05, P130=7.14264086072163e-05, P131=5.9299053901863844e-05, P132=0.00011212512741203111, P133=0.00010650312128737911, P134=9.942859538811022e-05, P135=0.00010698112819802935, P136=3.7274137510151624e-05, P137=8.771479637188087e-06, P138=4.353616114165627e-06, P139=2.5958232828301628e-06, P140=1.9703383406791122e-06, P141=2.890378774875487e-06, P142=3.3540815118222765e-06, P143=9.65264236609332e-06, P144=1.3236395364830969e-05, P145=1.7941912814492663e-05, P146=1.9908827271483765e-05, P147=1.8964434666053998e-05, P148=1.7705427866257185e-05, P149=1.23860669277364e-05, P150=1.0095961370887263e-05, P151=5.644405843845091e-06, P152=3.824592218707404e-06, P153=2.8777885475181765e-06, P154=4.3690508575307925e-06, P155=5.4511552194276405e-06, P156=9.528428341860453e-06, P157=1.6267615386295573e-05, P158=2.5941992121477015e-05, P159=1.942544617573596e-05, P160=2.1396654607211796e-05, P161=2.4012144398109318e-05, P162=3.0072060657005842e-05, P163=1.6504476075659045e-05, P164=3.5001826397176074e-06, P165=2.9197332580297786e-06, P166=7.5890863238174625e-06, P167=9.490843463583119e-06, P168=8.016351024141971e-06, P169=6.599917958349363e-06, P170=2.530102076206229e-06, P171=5.794425426438009e-07, P172=1.6410265549152282e-06, P173=4.339712482203616e-06, P174=5.816726419259693e-06, P175=6.256453247594719e-06, P176=5.409675026395637e-06, P177=3.949746178339654e-06, P178=1.8244891089917699e-06, P179=1.789365989767888e-07),Row( P0=-679.9842598174694, P1=90.6250768719676, P2=-9.686849077983602, P3=3.082180206972865, P4=-10.92506719946719, P5=-12.59555698519174, P6=2.894432446819441, P7=6.883993013664576, P8=1.2684031020428796, P9=-11.76463210052419, P10=-1.7674686014146315, P11=-6.76647208875658, P12=-5.8183439441164815, P13=-4.344169734530152, P14=0.24418699656089565, P15=-6.0698500607588, P16=-2.583751840595182, P17=-4.319001570687065, P18=-8.374735509453053, P19=-1.4704293430236501, P20=-12.000071920236248, P21=-3.9251170641894912, P22=-4.505883696143143, P23=-7.663064363950001, P24=-7.025984887000738, P25=-4.012146824649606, P26=-7.801167641831067, P27=-2.509761292397908, P28=-5.538966055420109, P29=-0.8133011474966602, P30=-4.881302200874332, P31=6.124051316971834, P32=-1.1850494624884118, P33=8.038848810842811, P34=-1.1878898282500263, P35=4.140638955805659, P36=2.2318416515172586, P37=3.873650352583256, P38=0.3475397440590946, P39=1.3413079076042977, P40=0.7414870573955821, P41=0.6093282888238589, P42=0.6657604614337195, P43=0.6633034750459635, P44=0.5641300087723381, P45=0.6139946013601346, P46=0.6393153600488365, P47=0.6679580643246745, P48=0.6995526175386819, P49=0.7017494184065626, P50=0.7116120698730951, P51=0.7610154808888491, P52=1.088535963906862e-05, P53=1.640750474839297e-05, P54=9.7056868327362e-06, P55=1.7692342089491485e-05, P56=0.0028324084563230477, P57=0.02881475740798418, P58=0.02147130579085709, P59=0.012997332646068865, P60=0.0032719162039819435, P61=0.00020050091253464016, P62=0.00012126585969013044, P63=0.0004758618738950668, P64=0.000547670179823584, P65=0.0012878879562601337, P66=0.0041099142759417435, P67=0.005277178036253634, P68=0.0027225627795745017, P69=0.0014613999035364836, P70=0.0018804629749827717, P71=0.000823623512733714, P72=0.0005537434438233139, P73=0.0013639456700840424, P74=0.0018899015347023853, P75=0.0013239826638068429, P76=0.001946316031788666, P77=0.002485392981306457, P78=0.002052820054152105, P79=0.0016150397115764555, P80=0.001877160703471826, P81=0.0043954609606070595, P82=0.006001186382497118, P83=0.0036532534146220034, P84=0.002894418872513649, P85=0.0021854008491082383, P86=0.0013961181945797065, P87=0.0020265480941673046, P88=0.005912254315585197, P89=0.0017004001879929133, P90=0.00287234626399221, P91=0.003848512193620736, P92=0.0013162101502550865, P93=0.0009850915267855262, P94=0.0005758683631441672, P95=0.0013517700224399693, P96=0.001867928930751363, P97=0.0006333095360299485, P98=0.00034547994987538954, P99=0.00029282467933008276, P100=0.0017125286616575528, P101=0.002679880696296173, P102=0.0002951501993602591, P103=8.62401435945479e-05, P104=0.00019559216160957766, P105=0.00015737569393958585, P106=8.292828140696753e-05, P107=0.00021647499201722664, P108=0.0006274359557540754, P109=0.0004292068665999061, P110=4.9675778159237264e-05, P111=8.526275186727621e-05, P112=0.000301933005641525, P113=0.00014348712567866998, P114=9.32860725376896e-05, P115=3.775702234582412e-05, P116=3.820200574794675e-05, P117=4.084639016715678e-05, P118=3.3877311116428045e-05, P119=4.312632668516545e-05, P120=5.0327121726266546e-05, P121=2.5538770087431013e-05, P122=3.481953701885437e-05, P123=4.017350573508246e-05, P124=5.913330744917167e-05, P125=5.132028466457266e-05, P126=4.7828419077447586e-05, P127=2.0636813885439224e-05, P128=2.5459370939064517e-05, P129=1.7416073637269158e-05, P130=1.605842627437749e-05, P131=1.7825676887707058e-05, P132=2.6772008434154505e-05, P133=3.265626964808386e-05, P134=2.7592399994410674e-05, P135=3.5673884470200085e-05, P136=2.1012883636033967e-05, P137=1.2060159886279048e-05, P138=8.630034066474073e-06, P139=6.511486440993603e-06, P140=3.4359228286310254e-06, P141=4.5403264773448404e-06, P142=5.7550000834986664e-06, P143=1.094177228201046e-05, P144=1.538027427001443e-05, P145=1.6005031822779648e-05, P146=1.490574381736394e-05, P147=9.148212816453368e-06, P148=6.24720341433804e-06, P149=4.52583756106955e-06, P150=4.5345213552489455e-06, P151=4.853977870563018e-06, P152=4.728730571629418e-06, P153=2.0881063585794666e-06, P154=4.403096009457647e-06, P155=7.181926580448838e-06, P156=9.18645015992373e-06, P157=1.732951279295999e-05, P158=2.177432903496969e-05, P159=3.477132844666425e-05, P160=2.613885857401274e-05, P161=3.002669940256908e-05, P162=1.6062864007942303e-05, P163=5.520163836364892e-06, P164=1.4277050165363469e-06, P165=3.0704017621407744e-06, P166=8.258908294804225e-06, P167=1.0424702386628464e-05, P168=1.2739945727849631e-05, P169=5.958479670605078e-06, P170=1.4776488189956685e-06, P171=7.557935846628943e-07, P172=2.2767787201828548e-06, P173=2.9983616196099706e-06, P174=3.7878184696049644e-06, P175=4.903630955821556e-06, P176=5.514994984793374e-06, P177=5.4940335680785835e-06, P178=3.0302945630071353e-06, P179=3.4062913071943995e-07),Row( P0=-669.6375983681385, P1=75.94132826279923, P2=-9.136880291297713, P3=16.933753105328492, P4=-6.063040538470543, P5=-6.3704022943526395, P6=3.075442062498989, P7=6.779403568048846, P8=4.572339111106207, P9=-4.178549117944805, P10=0.572685663335443, P11=1.6773212222959792, P12=-4.04060547967539, P13=0.29648495737521857, P14=-0.71926733801498, P15=-3.4887593442157976, P16=5.679320425768729, P17=-0.3749643608135022, P18=-3.8354900371637295, P19=3.1204033045537654, P20=-5.264816474375917, P21=2.2088323040864504, P22=-1.2236396295726364, P23=-3.6389226183843317, P24=-2.981637362983022, P25=-0.3235820770309431, P26=-2.07623723646338, P27=0.20825511509815048, P28=0.548110850016219, P29=5.125957683854026, P30=-0.09674192718037809, P31=8.556907541259381, P32=0.5884849711148775, P33=4.683341594121897, P34=-1.726178718937078, P35=4.248397689980788, P36=5.12034093162283, P37=6.421984407727792, P38=0.49799447177098316, P39=2.6489086173449183, P40=0.7510670737323906, P41=0.7084151360536524, P42=0.7486630749217373, P43=0.6732560681163504, P44=0.630718733781304, P45=0.6944372538941961, P46=0.7215832141849094, P47=0.6955925572481126, P48=0.7036666436941094, P49=0.7712523375317806, P50=0.7692468049667025, P51=0.758444251317681, P52=8.797003157258306e-05, P53=8.592387479177389e-05, P54=3.366247630131798e-05, P55=1.6855084225717015e-05, P56=0.00047364469513316566, P57=0.003406530531081713, P58=0.0043942547831402725, P59=0.0054417796516322905, P60=0.00512428973964055, P61=0.0018363308848670565, P62=0.00044911958014496456, P63=0.0004682777844043711, P64=0.0005354003416171713, P65=0.001732426999098584, P66=0.002802049418405549, P67=0.0035557529294277844, P68=0.003735399325064736, P69=0.006180504097109923, P70=0.00646321475008528, P71=0.0019487111792437577, P72=0.0006121413780825627, P73=0.0010570762052223065, P74=0.00110492781954171, P75=0.0016684570944656468, P76=0.003239153067504625, P77=0.006543479651627544, P78=0.01515928594290956, P79=0.015031752526891396, P80=0.004969875012794769, P81=0.0018100742290801967, P82=0.0026307160724037292, P83=0.003883428971505172, P84=0.0058091785928533505, P85=0.007631125478322934, P86=0.004209691522091219, P87=0.004088755371609699, P88=0.0053468794659056445, P89=0.0019130919185547917, P90=0.0006268785257352453, P91=0.0017028286659393113, P92=0.004192045447266799, P93=0.002721334848906134, P94=0.004251473564144732, P95=0.0035096658850855265, P96=0.0016025216256846432, P97=0.001791302585423296, P98=0.003502668570126135, P99=0.0067813202333276545, P100=0.0073165995763524345, P101=0.001958324174775926, P102=0.0011494554962719538, P103=0.0016099970625034232, P104=0.000789620944441795, P105=0.0004839058103476228, P106=0.00013450896028064957, P107=0.0005998963672169735, P108=0.0014789668686629735, P109=0.000963297559145066, P110=0.0002751828590624705, P111=0.00022471857272458276, P112=0.0007590279688588862, P113=0.0006739869056537186, P114=0.0006704518636589438, P115=0.00018362838028601817, P116=0.000322539285141501, P117=0.0003700470191662119, P118=0.00011415618344875884, P119=0.0003222067083633896, P120=0.000277979352556693, P121=0.00025139216606256265, P122=0.00029848584349166505, P123=0.0005082506313108242, P124=0.000330581612726698, P125=0.0004747219913958205, P126=0.000217848637402075, P127=0.0001172750394130476, P128=0.0001951003411967077, P129=0.00020139720591790002, P130=0.0001777910968015004, P131=0.0001337107290224918, P132=0.00021004606550191842, P133=0.00015596215528710123, P134=0.0001099229917861868, P135=9.021861735129866e-05, P136=9.47742986688086e-05, P137=5.738468530180945e-05, P138=2.1901069297669724e-05, P139=1.5728405576505996e-05, P140=9.098176281904428e-06, P141=1.333384000072747e-05, P142=2.4386178597898328e-05, P143=3.572427547500021e-05, P144=6.43265292871256e-05, P145=5.928633621755409e-05, P146=6.442921629745936e-05, P147=5.444473594367794e-05, P148=4.296562750660443e-05, P149=3.842511449208279e-05, P150=2.8124912532747405e-05, P151=1.6092213899380072e-05, P152=8.576883045368058e-06, P153=7.527088820620145e-06, P154=9.064067249961198e-06, P155=4.3177729883526734e-06, P156=4.357159911937553e-06, P157=7.2753430717671405e-06, P158=7.099626944844156e-06, P159=8.445657734133066e-06, P160=1.2977109526039948e-05, P161=1.2903926318474642e-05, P162=1.3329535239518257e-05, P163=7.883785090485036e-06, P164=3.1674974439094117e-06, P165=2.883336386647687e-06, P166=3.3655744825627424e-06, P167=4.846044732805861e-06, P168=5.66359122929876e-06, P169=3.870050363456055e-06, P170=2.6748724300587554e-06, P171=1.193240118128138e-06, P172=1.7640793314466935e-06, P173=2.7367128765974074e-06, P174=3.3626803331713286e-06, P175=4.753932414464337e-06, P176=4.050745363010199e-06, P177=3.7601818370255195e-06, P178=2.0464027353751677e-06, P179=2.921094286063334e-07),Row( P0=-593.9868794779476, P1=59.2606509103136, P2=3.708172537951708, P3=4.752146291980183, P4=-14.596214695731014, P5=-6.875671315405807, P6=7.244058181710987, P7=0.17442043395756726, P8=-2.9618338580200314, P9=3.0654868069672716, P10=-3.904446270939369, P11=-4.629851924374351, P12=-0.2668140556352001, P13=-4.9919448763083665, P14=-0.401965594460073, P15=1.5054868059316642, P16=-0.7119170437920693, P17=2.741300998674894, P18=-1.772550027501236, P19=2.544484118322653, P20=-1.356777316754785, P21=2.177813605644672, P22=3.9514372172190035, P23=3.7567905489224205, P24=3.23819335549398, P25=7.026933729647907, P26=3.0560895231884513, P27=4.375920086852242, P28=2.859020928916395, P29=2.944759404548546, P30=2.2330268072328523, P31=6.030013816870137, P32=1.520205398333924, P33=5.312598585296131, P34=1.52806403982946, P35=1.6949667106692603, P36=3.0750703788531304, P37=3.389035923251601, P38=1.639823250394797, P39=2.5202544329405914, P40=0.705562740670251, P41=0.6845788940460537, P42=0.6714980392150206, P43=0.6945276241808325, P44=0.7055566430613106, P45=0.7663063974716837, P46=0.7534187931189927, P47=0.7061874326691341, P48=0.7125913158901159, P49=0.7211017513556627, P50=0.7047505967413913, P51=0.7029078357013209, P52=0.05354875281561873, P53=0.007216346024867278, P54=0.0002754758426914314, P55=5.9040874906865947e-05, P56=0.0009581191238624667, P57=0.005196702998788437, P58=0.006377486161979697, P59=0.012148722918514269, P60=0.01870153501443429, P61=0.05388634105292394, P62=0.07863682059284748, P63=0.03478905638658126, P64=0.022740246020678607, P65=0.017286200221996315, P66=0.0076733968371537654, P67=0.007402368618801652, P68=0.007982421949512621, P69=0.010992832774707771, P70=0.011920502727122476, P71=0.0322611126808501, P72=0.08350064348335189, P73=0.09455122408346868, P74=0.03501132882598509, P75=0.007958521634396707, P76=0.011589033562836535, P77=0.03130911099109767, P78=0.04235483981274105, P79=0.029593549631724846, P80=0.013882208835865245, P81=0.016430425399299953, P82=0.028460589560824968, P83=0.03283689294538799, P84=0.0373222644078979, P85=0.021444961347354304, P86=0.014670951922381601, P87=0.011050775188354092, P88=0.01463658287358921, P89=0.017220558991188486, P90=0.01194095917464781, P91=0.0060308372750365515, P92=0.006753813729855633, P93=0.007347418496506805, P94=0.018621929944723323, P95=0.01769914502113576, P96=0.028300205566014862, P97=0.015169598006393411, P98=0.012454359449254358, P99=0.013634103127467599, P100=0.005077281672160608, P101=0.006953145159883794, P102=0.009874186481024809, P103=0.006426995710813805, P104=0.001519928133984546, P105=0.0010567964721749908, P106=0.0025886173574947743, P107=0.0027788305930046622, P108=0.0013280216227913842, P109=0.0014760776561588008, P110=0.0011795585306369836, P111=0.00045457554744619816, P112=0.0005141313683447588, P113=0.0010506081050923504, P114=0.0009662164231542855, P115=0.0007326800521083201, P116=0.0008841980460089112, P117=0.00035134769611720154, P118=0.001206020706459533, P119=0.0008706333605703641, P120=0.00047141746278151733, P121=0.0004742292887636736, P122=0.0008776879790623949, P123=0.000538422904184014, P124=0.0005902396617961779, P125=0.0006625449977191191, P126=0.00047858600359349055, P127=0.001182690492799573, P128=0.001861160313739566, P129=0.00296248905496825, P130=0.001200187561118686, P131=0.0015303932915289096, P132=0.0014641502341002047, P133=0.0019296799920718473, P134=0.003537413775228076, P135=0.00616707299593221, P136=0.008372536501838609, P137=0.005544844221971614, P138=0.002987047156572723, P139=0.0023251788874410883, P140=0.0033739882164419443, P141=0.005530489171297837, P142=0.006357833824614941, P143=0.00561149305882638, P144=0.0018397415618368958, P145=0.0011669288861524323, P146=0.00033395441158088533, P147=0.00011662495339763654, P148=0.00011122814338246761, P149=0.00018293791998531396, P150=0.00020671935772379884, P151=0.00014468758413161968, P152=0.00022630865659321948, P153=0.00032691448061642115, P154=0.0003113604627512233, P155=0.0002711628452388974, P156=0.0003258462720333293, P157=0.0002617231038155052, P158=0.0002867411430316441, P159=0.00026366054220278113, P160=0.0002729940230007666, P161=0.00018256005890286185, P162=0.00013599882823963347, P163=0.00016431535646718897, P164=0.0002655061356974325, P165=0.0004003722313675977, P166=0.0003741647287108093, P167=0.0003306152980039502, P168=0.0004182724067998893, P169=0.00021406121414772858, P170=9.161666437229323e-05, P171=6.893537612657038e-05, P172=0.00011271281803005102, P173=0.00019261368351295626, P174=0.00022240891208917893, P175=0.00018851505560707742, P176=0.00013515496147202408, P177=9.716828146214529e-05, P178=4.853348998299934e-05, P179=6.785345423394868e-06),Row( P0=-599.6687031907908, P1=55.07090264426691, P2=6.148375039089334, P3=-5.281840264843116, P4=-17.797463975381376, P5=-13.127195808263707, P6=-0.5881565269751473, P7=-7.957963719836756, P8=-8.523999748844961, P9=-2.283786866593074, P10=-9.372676609298484, P11=-10.38912436336598, P12=-5.9670011425330465, P13=-8.830526030894672, P14=-4.57075378564473, P15=-2.1075690893787438, P16=-1.7488668639441178, P17=2.7949787311846035, P18=-1.884846936213421, P19=4.746370629723782, P20=-2.0581452383590775, P21=3.0371515938768106, P22=0.8464533626899804, P23=-0.8182229088989502, P24=1.5008374148534651, P25=5.560671826555945, P26=2.6925407954344474, P27=7.9455458738769265, P28=4.857334000890391, P29=4.527522155547503, P30=2.1778227927386977, P31=4.706881107753237, P32=0.9527082704797657, P33=5.293562435256909, P34=0.32152262837572143, P35=3.6606233424306986, P36=2.8660161457024493, P37=4.7027953873918396, P38=1.178947771157738, P39=1.447683241947037, P40=0.6065110352132338, P41=0.5766731430929799, P42=0.6217964902522546, P43=0.6058736280901711, P44=0.6257870829431073, P45=0.6364916029655128, P46=0.6290618720572865, P47=0.6184858734002617, P48=0.6316952589665232, P49=0.6827257631884451, P50=0.6882081747850933, P51=0.640472830793728, P52=0.000227190933353015, P53=6.640875648723015e-05, P54=1.0638980669524833e-05, P55=8.491032445137508e-06, P56=0.00035871152515731466, P57=0.0023684908708566793, P58=0.0029526498733895095, P59=0.008780901012414107, P60=0.026718831339225063, P61=0.04164792765343715, P62=0.042590050513345426, P63=0.046730504753236445, P64=0.03518994722760447, P65=0.07045635961928355, P66=0.05060444137590786, P67=0.009177899059654401, P68=0.00674131063662462, P69=0.01420368803322965, P70=0.0537756665925797, P71=0.048070037831749346, P72=0.008901830553272648, P73=0.009003831821657763, P74=0.011185238527101819, P75=0.012013047606627044, P76=0.01282837145076073, P77=0.028161346349754566, P78=0.05395108475274227, P79=0.09270498166104453, P80=0.11937375813328452, P81=0.06678558362378893, P82=0.03543934865840989, P83=0.041140828042069644, P84=0.041627750018857, P85=0.045119068356814125, P86=0.03187721050529317, P87=0.02182950959103963, P88=0.06858342751999769, P89=0.04436992404340736, P90=0.027197234103236094, P91=0.029138146714968383, P92=0.02038795123613537, P93=0.008581868270448748, P94=0.020137497793393477, P95=0.046276336536460475, P96=0.026774378482325622, P97=0.016840238237184002, P98=0.0107508513826599, P99=0.00825559774542038, P100=0.007127339820736983, P101=0.010019410236380866, P102=0.0053963370038889605, P103=0.00284293670237908, P104=0.002696221838742041, P105=0.003034378488753724, P106=0.003695374009511962, P107=0.0026311015021785148, P108=0.0016340140122614014, P109=0.0017600621722710628, P110=0.0021258308171651554, P111=0.0006317215007199012, P112=0.0006827662382525675, P113=0.003970303051096603, P114=0.004639424048835658, P115=0.0007354653202276617, P116=0.000499503891882454, P117=0.000560966988325397, P118=0.0005800184614803278, P119=0.00024042096938061861, P120=0.00012501756452835275, P121=0.00014313197685871733, P122=0.00027789219484894743, P123=0.00015196234697975212, P124=0.0004303887487068462, P125=0.000242759892471392, P126=0.00038571471253117057, P127=0.00044896937241400026, P128=0.0009807093443411343, P129=0.0005190717244598534, P130=0.0012308497342348372, P131=0.0009564697199133099, P132=0.0010187224712290223, P133=0.000997653309997774, P134=0.002315396534562948, P135=0.0022278833375872476, P136=0.003256252621135106, P137=0.0019197643415294582, P138=0.0010548866676214628, P139=0.0007743523734965628, P140=0.000671940694189594, P141=0.0010051420570042209, P142=0.0012217122507435718, P143=0.0008890733383214868, P144=0.001487620189766338, P145=0.0022258363990796007, P146=0.0007125483529085263, P147=0.0003067408160760827, P148=0.0001281606364924125, P149=9.517051124362754e-05, P150=7.52082387631055e-05, P151=5.7161212104473935e-05, P152=7.438395662371893e-05, P153=0.00015312941195012918, P154=0.000378765568204693, P155=0.00047111453269601595, P156=0.000898400758162618, P157=0.0014543024750320946, P158=0.0015588816009229857, P159=0.0011340725956850545, P160=0.0015662029750000564, P161=0.001663273237181161, P162=0.0010115992060168093, P163=0.0007965185352393319, P164=0.0005164464815037382, P165=0.00038029087684782897, P166=0.0005136236772561804, P167=0.0008836885945257315, P168=0.001098230869316217, P169=0.0009181060042244017, P170=0.00090357812695733, P171=0.0007851242027390616, P172=0.0005392621279135502, P173=0.00045591315957146765, P174=0.0005436838473194659, P175=0.0007142125513395758, P176=0.0007594523361247403, P177=0.0006006493039508161, P178=0.0002794495894731513, P179=3.116389055657467e-05),Row( P0=-487.18548452000164, P1=13.38893696411798, P2=-29.06098409054822, P3=-13.538307724923639, P4=-22.54141387703093, P5=-19.85519177553739, P6=-3.670894643094509, P7=-4.406564383709102, P8=2.0124343887330047, P9=1.0928954341863182, P10=6.438604267819448, P11=12.078691167279912, P12=8.88326160062562, P13=9.190099838672142, P14=1.2394438878994074, P15=1.1091231809903817, P16=2.6233254725565507, P17=4.498969966210085, P18=8.481864788788442, P19=6.1693073125811555, P20=-0.1926420566460352, P21=5.850123245164016, P22=4.276495185686796, P23=8.288369062166534, P24=2.9944829224900897, P25=2.5901716968570065, P26=1.011010559017437, P27=5.40498120083875, P28=4.6406334341924165, P29=2.5218780698215033, P30=0.37064048424948787, P31=7.255648008456221, P32=1.7774668462744379, P33=2.6561993718159265, P34=0.898747919814948, P35=3.323899608388997, P36=0.10251910860778733, P37=1.0894831474219522, P38=1.459461680349347, P39=4.599231149623833, P40=0.6389994753253748, P41=0.6034643768704755, P42=0.6216216688071923, P43=0.5773784185958405, P44=0.5895384452457911, P45=0.6339493923576984, P46=0.5956646402807675, P47=0.5946573252071742, P48=0.6713693002867431, P49=0.5970374118674447, P50=0.6001553220225646, P51=0.6365750347144493, P52=0.007226198231372897, P53=0.0008726941124893929, P54=2.206447865629026e-05, P55=3.6120454778160163e-06, P56=2.041009460727835e-06, P57=6.351908700403019e-06, P58=1.3884115417569768e-05, P59=3.0777434299479084e-05, P60=8.363471336486492e-05, P61=0.0004523757493323435, P62=0.0009769778130340502, P63=0.0016666827525509986, P64=0.0025793699063795554, P65=0.0057302768791739275, P66=0.01128915836933654, P67=0.014971763965149706, P68=0.015395189430684466, P69=0.050258771151300646, P70=0.2154114214512362, P71=0.2568072633996121, P72=0.19518194902392108, P73=0.23674528738212575, P74=0.2759172313853567, P75=0.4238325532389723, P76=0.7521830077472571, P77=1.4597123546615867, P78=0.7682072984382124, P79=0.2971973976857127, P80=0.2659466411683981, P81=0.4168703938987686, P82=0.9270051515724195, P83=1.3606805173263228, P84=1.35501509589684, P85=0.5535353140419781, P86=0.3095989806104844, P87=0.26983842876477443, P88=0.43385306715059435, P89=0.42926163013990615, P90=1.0767529533175297, P91=2.306923437587553, P92=2.6163360533800684, P93=1.2830912241004329, P94=1.592721856674009, P95=1.3625563306320987, P96=2.0397814178930624, P97=1.9925457056575764, P98=3.484218187035684, P99=1.0707851925457048, P100=0.017554308137687904, P101=0.015318212846090518, P102=0.33323026474213574, P103=0.9855547129029831, P104=0.4295610070241583, P105=0.14820044800388, P106=0.0770663252391074, P107=0.15347067071362436, P108=0.35543097585315786, P109=0.208637546563974, P110=0.033252088367459486, P111=0.06168200699078444, P112=0.23932443385214847, P113=0.18149644902725928, P114=0.03552892142576681, P115=0.030415797829372572, P116=0.03703397223395741, P117=0.16345750183924332, P118=0.20968343272615236, P119=0.042945632728801385, P120=0.011987394478262744, P121=0.01082049985597058, P122=0.03230406224860451, P123=0.1518886682130016, P124=0.15872150348637434, P125=0.05424712954145417, P126=0.026637250449041194, P127=0.04375089615983766, P128=0.06853381426110398, P129=0.051950655708709985, P130=0.06667917062495206, P131=0.05751492375286226, P132=0.018327747944858055, P133=0.0578136516632028, P134=0.304067485181921, P135=0.243067943668695, P136=0.07596358856058158, P137=0.03764276638031528, P138=0.01924637325537025, P139=0.009352920972938722, P140=0.015008662542547478, P141=0.01353066319327363, P142=0.017712903230836616, P143=0.008490683839730107, P144=0.009531995470549826, P145=0.011421827609159755, P146=0.009811975599859782, P147=0.007656969319584851, P148=0.009055563179360682, P149=0.007518962979208506, P150=0.013520015084897747, P151=0.011471415317983602, P152=0.005277625726792161, P153=0.004360011099356978, P154=0.0026840701286007973, P155=0.0017824229989061846, P156=0.0014128549183239155, P157=0.0007581514459373727, P158=0.0008691834815583045, P159=0.0011987834229228323, P160=0.0014895329610951737, P161=0.0017454594584992106, P162=0.0015443364690753173, P163=0.0014307872761507232, P164=0.0012237834202669007, P165=0.0011521416564827355, P166=0.0012089975679136352, P167=0.0009691835122139071, P168=0.0010911108281912397, P169=0.0012692519811478649, P170=0.001259258503973017, P171=0.001093339802757154, P172=0.0010142683152068, P173=0.001775894480866035, P174=0.002117139921952, P175=0.0021838086744025926, P176=0.002596642960906253, P177=0.002047054450423729, P178=0.0010679663238246137, P179=0.00012056919572859031),Row( P0=-628.9192815863064, P1=67.7732469563005, P2=-18.224592194606313, P3=5.438618818383155, P4=-13.076827744868492, P5=-13.414546239195985, P6=-2.001272991384192, P7=4.010472234242672, P8=-1.579595167545502, P9=-9.129797591652393, P10=-3.103476476656961, P11=-2.591431123336575, P12=-8.446129820911146, P13=1.3541932726423993, P14=-0.20725117408943947, P15=-4.530839506211196, P16=3.801684699603597, P17=-1.919674829938444, P18=0.22524249794448803, P19=7.683452821381798, P20=2.5699924483683643, P21=11.648417095890963, P22=6.543397022725299, P23=5.517210457557062, P24=2.6294095733834033, P25=3.1726283798048938, P26=-0.15700554972019032, P27=1.2077554079212052, P28=1.852704900870855, P29=2.8352406978696942, P30=-2.8073963425233477, P31=1.6512372954399372, P32=-3.663311640746338, P33=1.878153976538122, P34=-1.7013421022129411, P35=1.4285411633788503, P36=0.6540009754921812, P37=-0.41337066441031756, P38=-2.706674496192676, P39=0.8947614717109901, P40=0.653362182160888, P41=0.6082841080192656, P42=0.7397967486166257, P43=0.7224807357098777, P44=0.6302503564228444, P45=0.6738520356150598, P46=0.6961671599096754, P47=0.7238645025495061, P48=0.7081712862825521, P49=0.6552398624901452, P50=0.6487165161972354, P51=0.7005381405213755, P52=3.649460008541871e-05, P53=3.5702344367311906e-05, P54=3.088802739718393e-05, P55=7.708850799879829e-06, P56=8.550762857485725e-05, P57=0.0008942482042097168, P58=0.0006994103316385054, P59=0.000466940420657745, P60=0.002081833955399489, P61=0.00632226705560692, P62=0.023770053222858297, P63=0.05746482822500433, P64=0.04966092010896235, P65=0.017704581768103182, P66=0.012668291712178323, P67=0.011111619178114634, P68=0.0026258464123647533, P69=0.0013218495434988205, P70=0.003051837433639223, P71=0.001978796277468113, P72=0.0011845086125709277, P73=0.00292824814056324, P74=0.007699299301110483, P75=0.015123069934856724, P76=0.022151001406133942, P77=0.028229277652008996, P78=0.020626873372572328, P79=0.01866455397354298, P80=0.016170068301064093, P81=0.027047729717773514, P82=0.05731222948684676, P83=0.02173000624892405, P84=0.006639695624994572, P85=0.011772245168582766, P86=0.022010625111530856, P87=0.037660147069791594, P88=0.07673183807715833, P89=0.035865115874802095, P90=0.0163377059440676, P91=0.021495802583672615, P92=0.035735863549555734, P93=0.023101635659250927, P94=0.031643102457984705, P95=0.04201932868377824, P96=0.024071572901973027, P97=0.012510209097655878, P98=0.004675761813304057, P99=0.0020992267152218223, P100=0.0023683792775370176, P101=0.005268856730559317, P102=0.0071665534717406, P103=0.005537569496982615, P104=0.001428502908011965, P105=0.0005145652924607294, P106=0.0007329340555537851, P107=0.0027591161446207115, P108=0.004748079288533761, P109=0.0027915928335984484, P110=0.0019923523188785, P111=0.0004651634380123515, P112=0.0012436696430394095, P113=0.0013847676462448463, P114=0.0009067556489371763, P115=0.0004900231311592917, P116=0.0003943872700480433, P117=0.0008875372212910674, P118=0.0004965060635588438, P119=0.00026444450843137704, P120=0.0004409655435568201, P121=0.000854695501737013, P122=0.0009556388920296772, P123=0.000458146800843942, P124=0.0007116563446726547, P125=0.001547713274647309, P126=0.0010543995704167754, P127=0.00026390089044157157, P128=0.0004382164160946177, P129=0.0003572374017185177, P130=0.00020461295365470616, P131=0.0004388406981032946, P132=0.00048068365696793175, P133=0.0003876256348483158, P134=0.000368762017376969, P135=0.00033126611701056946, P136=0.0004222548583142369, P137=0.00011188901290997921, P138=5.452003742076274e-05, P139=3.666931700503762e-05, P140=1.6222991740506054e-05, P141=2.1824016615525294e-05, P142=4.2099298573364916e-05, P143=6.828681585555862e-05, P144=0.00014340485013119103, P145=0.000120429337394398, P146=0.00012481637194902822, P147=0.00010282954453947799, P148=0.00012911147594429098, P149=0.0001874767108878561, P150=8.920495611239338e-05, P151=7.832106837160784e-05, P152=9.90702234898727e-05, P153=4.135615126205622e-05, P154=1.6496937626874125e-05, P155=2.7302356462511364e-05, P156=4.285134071238237e-05, P157=0.00010106754539383825, P158=0.0001181749019972652, P159=6.631449884383271e-05, P160=0.00010653757199087676, P161=0.00011135727706906097, P162=9.32135097990754e-05, P163=5.931086315374601e-05, P164=2.5250123682981734e-05, P165=1.1637595447031224e-05, P166=2.37119128994025e-05, P167=4.0513138768413146e-05, P168=5.124524652741753e-05, P169=3.748275670948868e-05, P170=1.75295611011922e-05, P171=6.008214931091056e-06, P172=7.564191687550834e-06, P173=1.3505249948308558e-05, P174=2.379362075412939e-05, P175=2.3275769409791664e-05, P176=1.9996453005888192e-05, P177=1.4684106651105557e-05, P178=6.884806916042887e-06, P179=8.609788935200873e-07),Row( P0=-555.44743341218, P1=24.461972339349437, P2=-24.34904589436476, P3=-8.805100345872235, P4=-21.49961630208419, P5=-17.787412491526577, P6=-1.104187403219016, P7=-6.675159381874328, P8=-4.643162333230574, P9=3.930620368103601, P10=1.937343227060644, P11=5.836657611984269, P12=9.439288180395138, P13=9.008882327664914, P14=15.394699892967871, P15=14.368350544567265, P16=7.979070749885386, P17=8.249320862376424, P18=1.6898807964318416, P19=3.5674847980418036, P20=0.6287499824099763, P21=5.790732090478899, P22=9.097326268219701, P23=4.510866454670244, P24=-0.7690150983174552, P25=-0.6419937447116124, P26=0.3045037052136854, P27=5.159049195979946, P28=4.820417894423283, P29=2.860811440031702, P30=-1.087665661679131, P31=5.914860932624914, P32=1.4259458529978777, P33=3.754232081537151, P34=-1.060240382888216, P35=2.8080417003385447, P36=3.5776502911537804, P37=1.1986700321241943, P38=-0.9639192250288977, P39=2.3928347400215633, P40=0.5782900597462247, P41=0.6058868914556609, P42=0.7208652313543961, P43=0.6964267899089464, P44=0.567798034219643, P45=0.5408858426597551, P46=0.5204457263368159, P47=0.5518019211556994, P48=0.546795279102746, P49=0.5772701666475869, P50=0.6251893169692448, P51=0.6097855746199957, P52=0.005668813917247782, P53=0.0005382439810147091, P54=4.774499526654953e-05, P55=1.5292375563326622e-05, P56=6.708658851716847e-06, P57=4.186891259125503e-06, P58=5.1045077479532325e-06, P59=9.464741522246685e-06, P60=1.4290332165737204e-05, P61=2.3951192412262316e-05, P62=5.386439882471024e-05, P63=9.891887480680522e-05, P64=0.0003547888033270612, P65=0.006435890580564082, P66=0.019245225968388867, P67=0.03652048014777546, P68=0.10997618749579127, P69=0.33975612028216273, P70=0.4287318078263354, P71=0.24015868638001006, P72=0.12597384932618988, P73=0.06506971060296568, P74=0.11330375058280305, P75=0.3259999906573748, P76=0.26314036666904317, P77=0.08174641988346971, P78=0.08519285615300752, P79=0.0759015375898522, P80=0.06497825439503066, P81=0.11540905891073053, P82=0.13787169132266341, P83=0.09921192784273339, P84=0.07822582112228198, P85=0.06473856845069578, P86=0.0825445290701908, P87=0.1225143947859345, P88=0.1988051527611973, P89=0.20880566119331745, P90=0.09370227386201674, P91=0.03558215251449477, P92=0.016009304924188485, P93=0.006388510069106058, P94=0.01885220169340819, P95=0.3553625539735434, P96=1.3617162715953053, P97=1.1457776668862412, P98=0.40521980482488773, P99=0.19135881962406434, P100=0.12671364917855008, P101=0.1176721127644426, P102=0.04418858109532662, P103=0.013815941259754409, P104=0.009448922176741969, P105=0.011076734360473909, P106=0.051880148427995904, P107=0.0977967716107413, P108=0.25641279639654885, P109=0.2960706664439144, P110=0.03886045708569946, P111=0.004555540069710107, P112=0.0022537805525798663, P113=0.003388307631076631, P114=0.0015967945857520952, P115=0.0011359306993752713, P116=0.0024732493540208273, P117=0.009427017725981946, P118=0.007978817120353818, P119=0.0035078334520876053, P120=0.002734283020485563, P121=0.0018268236217553625, P122=0.0015755305722185246, P123=0.006236485833608466, P124=0.025959436982699133, P125=0.006112941617783332, P126=0.001086208742406006, P127=0.0008734933295591848, P128=0.001385355112625814, P129=0.003814324360640377, P130=0.003451170415930549, P131=0.0015795248962512482, P132=0.0023747193002776707, P133=0.010065022944125734, P134=0.009972925991218117, P135=0.00832127599375132, P136=0.005178434084508735, P137=0.0112479979107433, P138=0.004724161940654481, P139=0.003280727665507468, P140=0.0017423182269954392, P141=0.0024099077033486777, P142=0.0073637677456097825, P143=0.006063862382273487, P144=0.002546579191464234, P145=0.0026437850731968146, P146=0.0017897113167892928, P147=0.0010382492662102728, P148=0.00027989981993111656, P149=8.890862062773339e-05, P150=0.00016223549725672545, P151=0.0002289101569437261, P152=0.00014767270079017215, P153=0.00020791347266832557, P154=0.00017709211178777308, P155=0.0001787872076683663, P156=0.00016562197399514394, P157=0.00020312415137556578, P158=0.00029913485288241687, P159=0.00021176784175223174, P160=0.0001440422565254725, P161=8.920567683074874e-05, P162=5.745765758052529e-05, P163=3.358941356587265e-05, P164=3.6480708069891467e-05, P165=5.373191445900971e-05, P166=6.795243456482259e-05, P167=7.027803159046358e-05, P168=8.03021974793641e-05, P169=6.464532368022351e-05, P170=5.3694082478772846e-05, P171=4.499638575178329e-05, P172=4.0799021838821096e-05, P173=7.868392421662598e-05, P174=0.00012622425961518558, P175=0.00010573062247350111, P176=0.00014105616976283362, P177=0.0001645770454321979, P178=8.541487993654335e-05, P179=1.0534376655775015e-05),Row( P0=-556.4421841994485, P1=49.08208840995141, P2=-5.385214093987357, P3=0.14326046376586732, P4=-20.044279680040333, P5=-9.485261719769735, P6=4.2164536392009415, P7=-9.174061009680793, P8=-4.755942564223441, P9=0.6370901487710566, P10=-8.244540547058051, P11=-5.531218995281757, P12=-3.7044814626794844, P13=-5.306388256175128, P14=-0.9885181073863288, P15=0.8208886235779939, P16=1.6637706561832921, P17=5.843785744086181, P18=0.8007005522566969, P19=8.421210588098187, P20=2.129968199660735, P21=8.112230225055603, P22=7.329879025408923, P23=5.429769134127605, P24=3.5385336289856473, P25=6.882919615293231, P26=2.877869647838741, P27=7.4899950765664896, P28=4.4280653768531995, P29=2.7907339742134734, P30=1.6418125370718164, P31=2.1394497895826747, P32=0.6935498295749907, P33=3.6209038946574283, P34=-1.2229828021723421, P35=0.4720726691490587, P36=0.2779247934823293, P37=4.05269344066231, P38=1.0339591553566936, P39=1.1537582855607382, P40=0.6510708543795773, P41=0.6631836885024273, P42=0.6726393467509867, P43=0.6843392964680411, P44=0.6806929655596947, P45=0.739367816867958, P46=0.7193823116098941, P47=0.670688793650396, P48=0.6587461472884465, P49=0.7005912246391528, P50=0.6910173019568788, P51=0.6551449593327828, P52=0.011387520248328311, P53=0.0010546572863295003, P54=9.684714186132407e-05, P55=3.070888132701708e-05, P56=0.00021160162759613696, P57=0.0009057176555877016, P58=0.0005548390820565157, P59=0.0017582189333869807, P60=0.011477289021748575, P61=0.0412598241744008, P62=0.07944426389371523, P63=0.1024777492464214, P64=0.08867680827897666, P65=0.08181142154687987, P66=0.05246192715077781, P67=0.05022110865888016, P68=0.18700169589341403, P69=0.22686848861721412, P70=0.0711292089294365, P71=0.021208930821939934, P72=0.017195089727595373, P73=0.030989542222988328, P74=0.031196343026302358, P75=0.02713135282980558, P76=0.037723741062943156, P77=0.10384589324325892, P78=0.22973231511242886, P79=0.2920030205014001, P80=0.11446999388030818, P81=0.05899541028325787, P82=0.07634425693158554, P83=0.0797381589414546, P84=0.0873810217596678, P85=0.09680268782148135, P86=0.07797286307861863, P87=0.07048713593721104, P88=0.08760352258870381, P89=0.09140513136546573, P90=0.0780333828902133, P91=0.05310247060224141, P92=0.13592391396208617, P93=0.1626827983512476, P94=0.05328437544990474, P95=0.04206306743551273, P96=0.04410509215227342, P97=0.03745531245388422, P98=0.04936098333340107, P99=0.053148488540137144, P100=0.03017148981110116, P101=0.06027944525994465, P102=0.06832746399957035, P103=0.010939555023499813, P104=0.0036813691831409103, P105=0.0051559428442257875, P106=0.010336122161083173, P107=0.017137898404742266, P108=0.03460256357544763, P109=0.02967345420860647, P110=0.00622196734188046, P111=0.0036668664190508712, P112=0.004553583000022221, P113=0.006798387711525839, P114=0.005602728703166489, P115=0.0017552114755632199, P116=0.0019693596601662315, P117=0.0026015485333679834, P118=0.0018275851926254326, P119=0.000969954380879924, P120=0.0014584012801480996, P121=0.0011257953025871378, P122=0.0003587696747734955, P123=0.000606257705765795, P124=0.0008380594863749957, P125=0.0004210079523389452, P126=0.000552956577722537, P127=0.0017662636680824252, P128=0.0009695614362556764, P129=0.0008963045122125086, P130=0.0006373348012815846, P131=0.0015047097890989085, P132=0.0020446155668659587, P133=0.001426390245017177, P134=0.0030347524334941576, P135=0.004321810253814662, P136=0.0030974747632889718, P137=0.003609237266988356, P138=0.0031174904863754994, P139=0.0013269746276116877, P140=0.0013480166368846106, P141=0.0013143876876374027, P142=0.0019906962968781945, P143=0.0037403258407952373, P144=0.0031328566395298733, P145=0.002385663652841591, P146=0.0011340363356836065, P147=0.000504067497789478, P148=0.00032839675663840905, P149=0.0002631856612866927, P150=0.00011604522307926881, P151=7.980772277393191e-05, P152=8.434933918379592e-05, P153=0.00015374602329454293, P154=0.00030121844529168733, P155=0.0002944558525523697, P156=0.0003556025923384549, P157=0.0003688819157838408, P158=0.0004267659629596721, P159=0.0005287116921433035, P160=0.00038766823389450346, P161=0.0003174379872853346, P162=0.0002604940951096751, P163=0.00020492881114803684, P164=0.00012478435512213264, P165=7.731253215192773e-05, P166=5.4536216596476304e-05, P167=8.737225621013699e-05, P168=0.00012933286889556074, P169=0.0001258347555165341, P170=0.0001750081116676964, P171=0.00011131432180029748, P172=0.00012982959568631425, P173=0.00013322681476121714, P174=0.00016420385098794854, P175=0.0001650096487162705, P176=0.0002045033746916656, P177=0.00023629390639627924, P178=0.0001608219687920313, P179=1.5788483997608754e-05),Row( P0=-656.5187096849359, P1=73.37471350623724, P2=-2.3768885985707637, P3=4.160014959305227, P4=-12.385129435515177, P5=-15.843896341271023, P6=-5.113251016714014, P7=0.2783868454499302, P8=-4.3053136420016695, P9=-11.402031640306557, P10=-6.215861507213842, P11=-7.589966123344887, P12=-6.3984581788850035, P13=-4.022339802909058, P14=-2.701067025963751, P15=-5.995165292030205, P16=-3.6833714293453443, P17=-3.16242273382029, P18=-10.076540282411514, P19=-2.092998892033475, P20=-6.227355990484825, P21=0.10947244285987616, P22=-0.4558670393036635, P23=1.2177871186331508, P24=0.09615347512184715, P25=3.746168043274268, P26=-2.1542505014428674, P27=6.828917157400263, P28=-0.2952089550705201, P29=4.865884301636219, P30=-0.7515049186675328, P31=5.659039165851946, P32=-0.572016822206924, P33=3.0163168537392426, P34=-0.7997663785999474, P35=5.997193352376752, P36=8.179101195260133, P37=5.335178294253709, P38=2.0913398750893917, P39=-0.8999024249189977, P40=0.6023758049261246, P41=0.6026775182553115, P42=0.6773048310859122, P43=0.6386484897594595, P44=0.5778024259402215, P45=0.6678832598711911, P46=0.6882555960401293, P47=0.62583755480992, P48=0.5531371297891382, P49=0.5713175761183557, P50=0.6720644839387454, P51=0.6458294279210118, P52=2.1231277273242917e-05, P53=2.368018756177555e-05, P54=1.487225373872546e-05, P55=3.3746816075048816e-06, P56=0.00010542915777353723, P57=0.0013237679759249953, P58=0.0057611292046389154, P59=0.007630280407150239, P60=0.013099987444771385, P61=0.01915119799392651, P62=0.025635409368994458, P63=0.028348781096023713, P64=0.007226745143317165, P65=0.0016974723457135205, P66=0.0021659288683632434, P67=0.0013854770206199179, P68=0.001999786456453224, P69=0.005360459008792901, P70=0.01698955665852862, P71=0.010328743035076497, P72=0.0019827182583252736, P73=0.005376228996976872, P74=0.007478867650542268, P75=0.008406092461844424, P76=0.006613830653292835, P77=0.0039905486132625675, P78=0.002625846729754632, P79=0.0030926354520230824, P80=0.004648305867886073, P81=0.0021985419125969126, P82=0.0013230339136149005, P83=0.0024303794749548986, P84=0.004854898079213048, P85=0.008521771406632706, P86=0.007703337617015333, P87=0.004892813730690813, P88=0.003626175151625347, P89=0.001962512403236473, P90=0.0019416720865594594, P91=0.004080914722443032, P92=0.009316840454680559, P93=0.006486245270039313, P94=0.004337674932367879, P95=0.0033079290331614274, P96=0.0013553222691369887, P97=0.0014756878430494998, P98=0.0013969059701520628, P99=0.0020992815626169224, P100=0.0016048876201848901, P101=0.0019386230690196513, P102=0.0010831996250304685, P103=0.0005004664578716239, P104=0.0002604026248174388, P105=0.00010996650318636409, P106=0.00022726889133793595, P107=0.0005736309836083932, P108=0.0005515028846157831, P109=0.0004071011039302121, P110=0.0004443340494744201, P111=0.00011666937771963536, P112=0.00030897084352913445, P113=0.00036972241896648147, P114=0.00022391343244340743, P115=0.00010586703308001168, P116=0.00012264960051795438, P117=0.00013152555872596535, P118=4.951031627699158e-05, P119=3.790830358452604e-05, P120=0.00011757800542356073, P121=6.104003521168995e-05, P122=2.7814035838362726e-05, P123=4.450811977639772e-05, P124=6.156705606280193e-05, P125=6.218980052379798e-05, P126=5.984021632420173e-05, P127=3.427685576666036e-05, P128=2.912149728586413e-05, P129=5.2335982515832364e-05, P130=4.465012570724475e-05, P131=3.542790254669618e-05, P132=4.332336104855309e-05, P133=5.1719830281243556e-05, P134=3.900074319892993e-05, P135=3.9809929042639414e-05, P136=3.296363702892266e-05, P137=3.330896745704967e-05, P138=1.105680459257365e-05, P139=1.3619361494336077e-05, P140=8.536402251895424e-06, P141=1.0564298990828617e-05, P142=1.3914035559095723e-05, P143=1.6325357428090902e-05, P144=2.7651079122662554e-05, P145=2.7145346456543662e-05, P146=1.939126786960313e-05, P147=1.540906870768027e-05, P148=9.037677125829045e-06, P149=1.028531350536932e-05, P150=7.97009359344853e-06, P151=8.063117758226251e-06, P152=5.437309482929191e-06, P153=7.041146866868593e-06, P154=1.6449995901920994e-05, P155=2.1068165461915805e-05, P156=3.573869465582036e-05, P157=3.1678113353807945e-05, P158=4.687237206885904e-05, P159=0.0001311931425817207, P160=7.465811173850796e-05, P161=4.0131882857385464e-05, P162=2.1411556462838057e-05, P163=1.0074425041393434e-05, P164=9.420386898116199e-06, P165=2.34231003915335e-05, P166=4.581215098019803e-05, P167=4.880305669892073e-05, P168=3.705239420819078e-05, P169=1.293352044090364e-05, P170=3.376655744605719e-06, P171=5.259548515108353e-06, P172=8.408112251394276e-06, P173=1.156305592710829e-05, P174=1.4972518525974245e-05, P175=1.2239874990872015e-05, P176=1.1492308324010955e-05, P177=9.307764380693067e-06, P178=4.59586630675463e-06, P179=6.17674781487154e-07),Row( P0=-655.1955939519668, P1=76.38269857177873, P2=-16.94839079493489, P3=6.209934953806201, P4=-12.049827729584923, P5=-12.224913910473848, P6=1.9652797618699451, P7=2.6067502579187183, P8=-1.7446474723533534, P9=-9.029984448913034, P10=-2.6303368789063057, P11=-5.384251463443863, P12=-7.996572034470979, P13=0.4542380509347647, P14=-1.4257911363017455, P15=-3.0296764473500404, P16=0.5271412190239428, P17=-4.4321116391086175, P18=-3.5723096393675684, P19=-0.5455614653620638, P20=-3.2802976922107567, P21=5.3116422416372435, P22=2.3891187373825544, P23=6.840964704667838, P24=5.100751165449554, P25=10.341283688843966, P26=6.079941030457741, P27=5.108635365733777, P28=3.020543799443116, P29=2.258162481350229, P30=0.38410187132381696, P31=3.7174541200859648, P32=0.006397444331174474, P33=1.837889443134017, P34=-4.045150439488745, P35=0.5802954294106748, P36=1.7089674634355865, P37=2.1505143596532754, P38=-3.2473243194200205, P39=-1.5043511960409852, P40=0.6430984968291741, P41=0.6929471223188569, P42=0.7626530601618836, P43=0.7372512326279362, P44=0.7081799393450318, P45=0.736879664299025, P46=0.744990132616433, P47=0.6420842091214589, P48=0.5761646069965607, P49=0.6013139613883596, P50=0.666622765850207, P51=0.674525111882678, P52=0.00031380258371287845, P53=0.00011241936260172653, P54=3.237157737063824e-05, P55=1.1024993531979139e-05, P56=0.0004491630600016976, P57=0.0019703434032434216, P58=0.0007964657803758051, P59=0.0017311895368135833, P60=0.011578756857714432, P61=0.026038817039385882, P62=0.047819593390264634, P63=0.04806157007222356, P64=0.009081471863095481, P65=0.0007306097896390064, P66=0.0004638126898443778, P67=0.000742777490112191, P68=0.0015540315237391985, P69=0.006176589218231738, P70=0.011810690875200256, P71=0.011109607934700854, P72=0.007491087681276443, P73=0.01790844433039543, P74=0.026859589507992526, P75=0.029387710107421364, P76=0.01863637458110479, P77=0.0030702278841703615, P78=0.0018492736997977536, P79=0.004674118666664529, P80=0.008449672034646827, P81=0.025311592762380767, P82=0.03588482404205738, P83=0.022086528041504242, P84=0.025846832247219575, P85=0.037129745142301926, P86=0.03908947952185187, P87=0.022469946043817083, P88=0.007937884235774764, P89=0.008249716980154373, P90=0.007912362903858192, P91=0.005749497354261726, P92=0.007407725479495411, P93=0.011161667853568407, P94=0.0145406015922664, P95=0.005978560407371472, P96=0.005278149235762311, P97=0.004322722151988149, P98=0.0014115451504662134, P99=0.005363989235508477, P100=0.010836757573531211, P101=0.01171739143260949, P102=0.0043613369344200565, P103=0.0015340227901497736, P104=0.0003844637729141917, P105=0.0008364186234744368, P106=0.0016474470012603534, P107=0.0013506825883259203, P108=0.0005585002314869016, P109=0.001364474161041101, P110=0.0035735649617719646, P111=0.0008735832272826492, P112=0.0007197876483410864, P113=0.000420543772392253, P114=0.0009104271965801584, P115=0.0009585189982125238, P116=0.0007212797999201453, P117=0.0001517533785022682, P118=0.00043096453531643456, P119=0.00020190080727971885, P120=0.00017416228383694776, P121=0.00047148166028520937, P122=0.00040364534552949326, P123=0.00030744461072128564, P124=0.00073525110897024, P125=0.0010768921773093903, P126=0.0006001081429933482, P127=0.0004656741993668971, P128=0.0006458043194004206, P129=0.0005173792511159005, P130=0.000697152937328881, P131=0.00030528406697753985, P132=0.0006379809413289411, P133=0.00044111892385457677, P134=0.00040231328768493773, P135=0.00037684512203105003, P136=0.00028936101859067465, P137=0.00021624662938407886, P138=6.182010455646693e-05, P139=5.5952832454963485e-05, P140=2.3468614169000087e-05, P141=3.1640816096460824e-05, P142=4.0030272002859135e-05, P143=3.848532681574738e-05, P144=6.051513323424835e-05, P145=6.558527664751722e-05, P146=7.265017086968113e-05, P147=3.938415687003671e-05, P148=3.383624907062922e-05, P149=4.5623524123692296e-05, P150=6.912873453255414e-05, P151=6.508202372603624e-05, P152=5.7693501088484574e-05, P153=2.3613607358435537e-05, P154=1.26394815815323e-05, P155=7.929910977529688e-06, P156=5.942171802138578e-06, P157=6.367498681451335e-06, P158=7.062702645594574e-06, P159=1.9023281110468745e-05, P160=1.5204767583516703e-05, P161=2.1440628188234058e-05, P162=2.3139320997772555e-05, P163=9.152142712342057e-06, P164=7.383300389153257e-06, P165=4.5512491928695245e-06, P166=2.9546019334795026e-06, P167=3.5652677154877778e-06, P168=4.862735071899322e-06, P169=4.0082472225833054e-06, P170=2.07316408456962e-06, P171=9.996351169240977e-07, P172=9.650637392888973e-07, P173=1.912240309154339e-06, P174=4.851180870506998e-06, P175=5.5274568719402014e-06, P176=4.463820124244925e-06, P177=3.172036127394839e-06, P178=1.4463405084949023e-06, P179=1.503876106241423e-07),Row( P0=-701.3454957566455, P1=52.142502356897346, P2=-2.484198249685788, P3=-0.9697338582132263, P4=-14.002949959943061, P5=-10.068752942317728, P6=1.4496655927463693, P7=-1.9828513270815018, P8=-5.8648441026695, P9=-5.7953872620569244, P10=-4.9603265056842, P11=-7.648356079275693, P12=-6.45000071104502, P13=-6.186859977910239, P14=-5.122655439104824, P15=-3.4442737864609705, P16=-3.910577727942903, P17=-5.856567330763676, P18=-2.3517184409471614, P19=-4.214935455083926, P20=-6.878864384742574, P21=-1.362659341891673, P22=-3.583872508835127, P23=-0.8561143400023018, P24=-5.318826797614918, P25=1.9715305880930787, P26=-3.0131650023697887, P27=2.5803440332778154, P28=2.1918010677950583, P29=1.0647489112753403, P30=0.28307223516924185, P31=3.041952931647783, P32=0.1676678645248041, P33=1.7043669722186245, P34=0.30313859798714377, P35=0.19218613289355443, P36=1.4827927293706489, P37=1.5016858427715576, P38=-0.9014499409693006, P39=1.2537889884032052, P40=0.5821137800472916, P41=0.5817588704619348, P42=0.6270006157404386, P43=0.6316671826960429, P44=0.5929265876372957, P45=0.6066849131280853, P46=0.563231470369767, P47=0.5422098405011275, P48=0.550712673966211, P49=0.5612988944098332, P50=0.5793998227775348, P51=0.5645627340340248, P52=4.369834834396662e-05, P53=5.6348796446153486e-06, P54=1.1829661470106723e-05, P55=2.369902981383132e-05, P56=0.00029173268167724466, P57=0.0017575801108749983, P58=0.002648465913372987, P59=0.005113912676825903, P60=0.009561353315807413, P61=0.010580587899461738, P62=0.007957052623079086, P63=0.006073005431412907, P64=0.004349911941628177, P65=0.0011428504393086653, P66=0.0012906701896734855, P67=0.0025567427817154383, P68=0.0034192516879236845, P69=0.0060495350404177264, P70=0.009257632891755114, P71=0.004689586278499946, P72=0.0016761809309522196, P73=0.0017785237752047235, P74=0.001947041347383121, P75=0.002405420026611373, P76=0.0041527119015172505, P77=0.00706997291514288, P78=0.016141843563063717, P79=0.019844375417460362, P80=0.006986293307066291, P81=0.0039402511370008695, P82=0.005188922782395015, P83=0.007419798856226352, P84=0.009299140542352001, P85=0.008387962263537574, P86=0.007283303245921586, P87=0.005419473689905111, P88=0.008004476844421772, P89=0.003965518577599288, P90=0.0034878504336904245, P91=0.004674718810755292, P92=0.0034288989006708535, P93=0.0018806919916631957, P94=0.0028365008963475453, P95=0.0034208092276127324, P96=0.008483008796710052, P97=0.004075560613562557, P98=0.0016207220145908544, P99=0.0015473543065482947, P100=0.002875131523034441, P101=0.005077774002070984, P102=0.0009935214946980512, P103=0.0002986751879571302, P104=0.0004763572274982085, P105=0.000958684350923386, P106=0.000494315639795977, P107=0.0004432713967586606, P108=0.001738823337533703, P109=0.0011395795055781069, P110=0.0002552262726969217, P111=8.049667876306422e-05, P112=9.353428674226486e-05, P113=8.979829117357552e-05, P114=9.839703751364167e-05, P115=3.971022688697477e-05, P116=2.9224754258524948e-05, P117=3.2099342525751305e-05, P118=8.898718548116558e-05, P119=8.121208542439317e-05, P120=4.511949306382438e-05, P121=6.19655311846655e-05, P122=0.00014905201932184842, P123=9.103061797849363e-05, P124=0.00011685066262736451, P125=0.0001023212571117354, P126=4.8555658129417485e-05, P127=4.7193605813995235e-05, P128=7.184411603202937e-05, P129=4.849350667065778e-05, P130=3.806762534627655e-05, P131=9.750280046395204e-05, P132=8.4385743341544e-05, P133=0.00012178594727323269, P134=0.00019518220609042924, P135=0.0002554141725049416, P136=0.0002437860467724329, P137=9.268168312161852e-05, P138=6.544912435071637e-05, P139=4.114432154331246e-05, P140=1.957115135610992e-05, P141=2.718853429485417e-05, P142=3.0697742839633204e-05, P143=5.0631960923309826e-05, P144=3.121679058270083e-05, P145=4.2626761945053385e-05, P146=2.8369412419835085e-05, P147=1.2549637318563195e-05, P148=8.3168547242389e-06, P149=2.847565194592787e-05, P150=6.028371679805141e-05, P151=7.69158680629612e-05, P152=5.1399823384719896e-05, P153=4.38851054496018e-05, P154=3.9493380295568456e-05, P155=2.9009263358483512e-05, P156=1.5575247267304562e-05, P157=1.2103568971063822e-05, P158=1.0538367265199895e-05, P159=9.18751815861567e-06, P160=9.807482804724237e-06, P161=1.6689055140218243e-05, P162=1.7368306914501562e-05, P163=1.6424637134972874e-05, P164=2.7081671947886605e-05, P165=2.2652452569949445e-05, P166=1.473525998040957e-05, P167=8.820671227400827e-06, P168=4.351621834146887e-06, P169=4.28496877609817e-06, P170=3.0149667535467356e-06, P171=3.7647535198790427e-06, P172=5.478911411770744e-06, P173=6.741204154031412e-06, P174=8.413495079388876e-06, P175=5.378750366919689e-06, P176=7.401096458874133e-06, P177=6.264373574607088e-06, P178=3.7076636171447153e-06, P179=5.870828891613901e-07),Row( P0=-469.7658170562164, P1=12.723721154501087, P2=-28.47883157813983, P3=-11.875482629184447, P4=-25.348587985208688, P5=-26.85434806987841, P6=-5.507916627499048, P7=-6.145722991687105, P8=-2.9783716190002822, P9=-1.3670082085811504, P10=2.8436086444993363, P11=8.133528617372702, P12=5.572283354018484, P13=7.506545796899147, P14=1.0750808423112634, P15=1.0953547579098397, P16=1.5601418539086955, P17=2.7941434745540943, P18=2.0922290295955124, P19=4.273749573517546, P20=-0.7647256375224747, P21=6.487650786660842, P22=3.349226416789901, P23=8.117414954688721, P24=3.1232803312920985, P25=4.4505826410220335, P26=-2.433551558674873, P27=1.8690833535966227, P28=3.8621545583435712, P29=0.8739433999003552, P30=-0.43766545093285286, P31=3.967951444320324, P32=2.012012191450965, P33=2.26373357630325, P34=-0.2901759100674099, P35=0.9626760840915854, P36=1.1930590781885175, P37=0.446761656405576, P38=-2.7339210209731775, P39=1.0491237332337027, P40=0.6229382490360115, P41=0.6193954467438996, P42=0.642682940855077, P43=0.6109801596843233, P44=0.5977772003511826, P45=0.6907360829237735, P46=0.6909119306136732, P47=0.6419220316557285, P48=0.6152814886521745, P49=0.5430273131296844, P50=0.5747668789431198, P51=0.6260380986272985, P52=0.0010882134435209587, P53=0.00017990879696135005, P54=4.522616254346997e-05, P55=1.0330921369481146e-05, P56=1.0295330876137465e-05, P57=1.7853592261447205e-05, P58=2.9759430022629475e-05, P59=6.804301992440812e-05, P60=0.00014739152161720256, P61=0.00029345606419942935, P62=0.0008750589308829125, P63=0.001091461175238722, P64=0.0007137279133622335, P65=0.0021203651772500717, P66=0.008770326263167193, P67=0.021876336521399337, P68=0.035153767081754383, P69=0.0664106561971929, P70=0.16186444762005633, P71=0.14956901954723467, P72=0.28287854194371365, P73=0.9283626898955597, P74=1.3502208122835897, P75=1.3115247199566673, P76=1.6196519157835165, P77=1.4520962291804675, P78=0.5657687995548134, P79=0.3694698953447006, P80=0.20130789916136263, P81=0.19233096868793662, P82=0.4165882448596735, P83=0.5863110415167921, P84=0.8077316157702109, P85=1.0720950959138427, P86=0.7812403996784864, P87=0.46218113626968493, P88=0.4289431315653317, P89=0.2212531404396851, P90=0.22254456256573907, P91=0.5431637921116149, P92=0.842761843682537, P93=0.9404048982597459, P94=2.5748216179430696, P95=1.864120096374935, P96=1.101739232629317, P97=4.248177777483496, P98=2.8353084355770224, P99=0.11707192303307998, P100=0.026481618490146783, P101=0.02205110183521246, P102=0.09057669994389636, P103=0.17228086564055228, P104=0.1083367507225156, P105=0.03533958710940221, P106=0.04009130320832831, P107=0.17562243286012746, P108=0.10523297864423403, P109=0.08023070974952419, P110=0.0297476611175758, P111=0.013352549967757511, P112=0.04448907038092753, P113=0.12231488199588846, P114=0.09928433519431955, P115=0.06736025369382452, P116=0.1390957657342049, P117=0.03160150567117708, P118=0.024736150686183744, P119=0.015266304966192775, P120=0.007498874673498837, P121=0.01151699134963634, P122=0.10430301326940432, P123=0.09380281418064951, P124=0.06725824617141615, P125=0.059321886711376524, P126=0.0313705623840483, P127=0.08067224287752085, P128=0.09098592642046134, P129=0.0824606003833139, P130=0.0548678636362417, P131=0.040905798901861155, P132=0.15468950361762895, P133=0.23590443904391142, P134=0.11235410737648406, P135=0.13558602832369235, P136=0.15868246973260505, P137=0.0896788340795732, P138=0.04626371377963713, P139=0.013571113459565267, P140=0.033780350512248446, P141=0.023381920850432118, P142=0.013057615878643172, P143=0.07026924495662203, P144=0.05542322428165235, P145=0.00980930002687449, P146=0.014603812081421901, P147=0.009537866506504523, P148=0.004186761712059595, P149=0.008246067358504085, P150=0.008241922134967578, P151=0.009648439673394883, P152=0.007510065206465586, P153=0.002941689450049878, P154=0.002609512332829669, P155=0.00335584691125663, P156=0.0022713661153402143, P157=0.0019139189271766119, P158=0.0013038288464646533, P159=0.001647334050718024, P160=0.0016642054781036617, P161=0.002371976345831119, P162=0.0018711280687742076, P163=0.001971460119823506, P164=0.0033319127397808805, P165=0.002637475388657108, P166=0.0016598422449607252, P167=0.001249796687070752, P168=0.0010683024616373184, P169=0.0018158546300318418, P170=0.0018202189986695088, P171=0.003091730629693523, P172=0.003777216115008777, P173=0.0031371357744594615, P174=0.002844818257407673, P175=0.0031272854709267365, P176=0.002681627631633197, P177=0.0021937853099413232, P178=0.0011448691707931693, P179=0.0001563389822018487),Row( P0=-501.80194147699217, P1=58.02276640223882, P2=-24.88266302837931, P3=-2.073725489711874, P4=-16.094474091979578, P5=-19.135427398180468, P6=4.269215463584224, P7=4.640657031489302, P8=0.9162432016540538, P9=0.10621223400440047, P10=-5.459525004076961, P11=2.5256696543206814, P12=-2.7153778160556055, P13=4.841144522200024, P14=8.78393382275476, P15=6.653719615689708, P16=15.41335410370062, P17=13.954197917801588, P18=11.638066469094474, P19=15.0075806397517, P20=6.179035477303045, P21=11.195299607550014, P22=6.795029012393127, P23=4.91268654495642, P24=1.5995605113541462, P25=5.224113133242848, P26=3.5179062404956807, P27=4.0377551071805, P28=3.211901919979679, P29=1.4786191689972465, P30=1.6729934268430813, P31=4.527904007954706, P32=0.3712466189423014, P33=2.9610180120131675, P34=-1.7458881241847244, P35=1.3462674488530104, P36=1.1691657058144493, P37=2.0411968208450633, P38=0.3056371563838745, P39=2.1701458441338186, P40=0.6386775027205742, P41=0.6655332254750783, P42=0.6720939723004523, P43=0.6196671317157816, P44=0.6348802028647218, P45=0.5678321940036484, P46=0.5571513060191979, P47=0.5700015548527847, P48=0.5616708375311716, P49=0.6319566019099049, P50=0.6672852104150336, P51=0.655368987380462, P52=0.2445321349741415, P53=0.029902230079462876, P54=0.0013454416636079507, P55=0.0003133297518500145, P56=0.00010005086399847206, P57=5.4975687492989554e-05, P58=2.8303836897869256e-05, P59=0.0004562005585257348, P60=0.004788289254367878, P61=0.013675402876111523, P62=0.0381954883080499, P63=0.07952754489079593, P64=0.11271565436953276, P65=0.3655008176572844, P66=0.433065746871001, P67=0.3255927921950597, P68=0.39292930763173056, P69=0.2864074066592382, P70=0.21121502022993302, P71=0.10590562339370296, P72=0.015739764599458195, P73=0.013363438784360915, P74=0.022884255929445218, P75=0.03574213315913387, P76=0.06395732540753643, P77=0.15852358629463573, P78=0.32195713326746445, P79=0.5085506228524014, P80=0.5652661360121026, P81=0.38713922347253477, P82=0.3130684887553398, P83=0.2841798558697257, P84=0.20472510123263052, P85=0.16204664740969796, P86=0.14133019113014603, P87=0.17409217438766336, P88=0.20377629484330662, P89=0.14726951812978795, P90=0.20905538295197756, P91=0.2863476757928968, P92=0.4040292406028009, P93=0.16932941610747193, P94=0.28955188001687865, P95=0.2622472879950145, P96=0.20445372562951228, P97=0.19979965785423331, P98=0.16916097897300714, P99=0.16682711711104548, P100=0.10618062977236079, P101=0.07518850819922256, P102=0.028814453598596794, P103=0.024420262548913656, P104=0.019944320694103044, P105=0.006429066328941462, P106=0.009322767260563199, P107=0.026667425907450516, P108=0.02872852383549867, P109=0.023955618467848824, P110=0.02247372619678893, P111=0.029583126019975387, P112=0.011098206643969927, P113=0.01726038933593875, P114=0.009868295107128652, P115=0.005222081764370786, P116=0.014568381907553716, P117=0.02888870272695967, P118=0.030008768891402793, P119=0.006308202847844793, P120=0.004278587611719629, P121=0.02098537913183399, P122=0.02064720461906321, P123=0.009032555450926343, P124=0.023156688445215387, P125=0.01922279719689719, P126=0.011874063012966235, P127=0.0102880928169906, P128=0.026141735445372922, P129=0.04173757407933284, P130=0.02599002856467161, P131=0.03714785432989758, P132=0.034560859603266866, P133=0.021410491767502683, P134=0.024281025240541727, P135=0.022210546990015775, P136=0.014841026399142919, P137=0.009471896864153352, P138=0.0033198228479896234, P139=0.004146949151594812, P140=0.002661396071075197, P141=0.001752648480967484, P142=0.002428132660225298, P143=0.002614694278143405, P144=0.0036312198986247273, P145=0.0029490438914098263, P146=0.0019862571120481433, P147=0.0015313621334480164, P148=0.0014857775676333218, P149=0.0013999482265271113, P150=0.0015978647132175773, P151=0.0011806721918041633, P152=0.0012352954181772493, P153=0.0005217681085504265, P154=0.000448229460261382, P155=0.0003429985036158318, P156=0.00027083353491221175, P157=0.0002454478606975979, P158=0.000210997608454816, P159=0.00020770190747284572, P160=0.0002500232175680623, P161=0.00028218302944755294, P162=0.0002690128473642016, P163=0.0002620055768064174, P164=0.00019023299873002406, P165=0.00013839175086100623, P166=0.0001213308047081196, P167=0.00014877251264612358, P168=0.00019616048650407988, P169=0.00017201512388110513, P170=0.00018888343813882667, P171=0.00013452766158380237, P172=0.00015787465935034989, P173=0.00016302439876949518, P174=0.0002138878786195961, P175=0.00023128858342733936, P176=0.00023431850305356475, P177=0.00022675718195157047, P178=0.00010504063658709591, P179=1.0711226599510141e-05),Row( P0=-680.8124622472162, P1=62.1346431730933, P2=-9.04354589601294, P3=-0.6397188149550126, P4=-16.024363305925448, P5=-11.407379830608615, P6=2.3632249876230422, P7=-1.6336209567687965, P8=-6.897475201661991, P9=-6.801260476837394, P10=-4.94681289415334, P11=-8.079639258990275, P12=-6.653662047038089, P13=-6.243587342263495, P14=-4.196627020075437, P15=-5.714814889028633, P16=-1.6664912563027343, P17=-7.2378157274495045, P18=-2.192977310374729, P19=-2.1767987696716316, P20=-4.337887503308264, P21=1.3462341571654115, P22=-2.2980046432810926, P23=-0.9640972687417867, P24=-5.997550046741098, P25=0.32869940327467584, P26=-3.6905486633217888, P27=-0.5322698651325806, P28=-0.8342996485380517, P29=-1.1598725489162192, P30=-1.0019297091832788, P31=-0.2993581018820057, P32=-2.6827137992538077, P33=0.31296179996053836, P34=-3.6024249762160947, P35=0.03088071449121507, P36=1.1006236261925062, P37=3.0821082069515926, P38=1.3177161781553992, P39=2.6774299011173963, P40=0.6743691673025172, P41=0.6880119133838759, P42=0.7483541162191947, P43=0.7668963970799728, P44=0.7109369086667489, P45=0.7349538529416659, P46=0.7008962906292957, P47=0.6487202466806494, P48=0.695194050242661, P49=0.7236432791739231, P50=0.7166078734931675, P51=0.688290757747184, P52=1.2935754208528481e-05, P53=2.8363297721057617e-06, P54=5.508716655652289e-06, P55=2.1217897564323464e-05, P56=0.0007997284381505146, P57=0.008029636424117687, P58=0.007426877724482788, P59=0.0030621479121202243, P60=0.0037406252025572527, P61=0.0059381351008948955, P62=0.006692063983702705, P63=0.00827265918934426, P64=0.007726290312697986, P65=0.00372761388672124, P66=0.002746702975351311, P67=0.0018574660066120203, P68=0.002171843530323747, P69=0.0023054545310878787, P70=0.003687386444592159, P71=0.004010941569818127, P72=0.004419335079042857, P73=0.004648282676721964, P74=0.003255403463022988, P75=0.0030070846972533364, P76=0.0053459520553675996, P77=0.010206364431284505, P78=0.008673123639977867, P79=0.007878822282554323, P80=0.00910111258981378, P81=0.00823324911039457, P82=0.006562865215872299, P83=0.005479401694653246, P84=0.007047034909909867, P85=0.007233040544825327, P86=0.004629216969694106, P87=0.005801930818197932, P88=0.01864307155857699, P89=0.015422449102870934, P90=0.00481307101258929, P91=0.0029923653791317745, P92=0.019664477947528704, P93=0.009656743999520032, P94=0.010221271081750768, P95=0.005507548176184496, P96=0.0015508440273161387, P97=0.014489069212378018, P98=0.02361455530307124, P99=0.0059938757000903104, P100=0.0030344594880175753, P101=0.003237220311591877, P102=0.0043072202574194795, P103=0.00255511936061639, P104=0.0005359315871781194, P105=0.00035590118633436606, P106=0.0019299163816514817, P107=0.0022779258567451168, P108=0.0008743530692717205, P109=0.0006278305736996996, P110=0.0006528004308673919, P111=0.00010088960925955305, P112=0.00011437315792488806, P113=0.0003900940047336707, P114=0.00023529647234225251, P115=6.379032720717535e-05, P116=2.7428983480740706e-05, P117=5.066028670893201e-05, P118=0.0002777425412019303, P119=0.0005125612244307344, P120=0.00012061443132202185, P121=0.00020396945763827845, P122=0.00032159918805747627, P123=0.000250347246822374, P124=0.0001921342874376179, P125=0.00021445544497427445, P126=0.00014779982811821219, P127=0.00014058525293122464, P128=0.00021442013524198647, P129=0.00027339436019385303, P130=0.00011371016387074073, P131=0.00024237442902307657, P132=0.0002582148869433506, P133=0.00024220187046505853, P134=0.00038413156833341577, P135=0.00032984641422159096, P136=0.0003766051763345243, P137=0.0001555748479002033, P138=0.00011445806299558043, P139=7.360563367916304e-05, P140=3.5394481588896346e-05, P141=3.729522028672515e-05, P142=4.1446140862016356e-05, P143=9.150357918986665e-05, P144=4.8358278036788844e-05, P145=6.109147028001483e-05, P146=3.30732293469892e-05, P147=2.5388770639360063e-05, P148=2.1712908149183385e-05, P149=3.144319767853377e-05, P150=5.9879080789644946e-05, P151=0.00010964852190594005, P152=0.00010461833938526652, P153=9.868409222230329e-05, P154=7.147971022923607e-05, P155=6.244431268917279e-05, P156=5.130285763188021e-05, P157=2.607170459281534e-05, P158=1.0615887434928398e-05, P159=1.5217664622407345e-05, P160=1.98607861641939e-05, P161=2.3163401922958062e-05, P162=3.1791544528667634e-05, P163=4.879740900998356e-05, P164=3.775551071745306e-05, P165=4.230274994893052e-05, P166=3.1417019139632303e-05, P167=1.8608263914091114e-05, P168=9.135942246089661e-06, P169=9.803425969776492e-06, P170=6.712703796373053e-06, P171=7.696600771980822e-06, P172=6.83698492091243e-06, P173=1.1328594722442372e-05, P174=8.464544876556024e-06, P175=7.863456732976083e-06, P176=8.39632114992202e-06, P177=1.1670250444877888e-05, P178=6.194224340890947e-06, P179=7.17298521648298e-07),Row( P0=-654.5786032349813, P1=48.87968650331944, P2=-20.288607469444628, P3=-4.557887955884198, P4=-19.27798505028092, P5=-21.742644685392907, P6=-5.44440637377108, P7=-1.6261350395876835, P8=-4.153389503637965, P9=-7.407951743561803, P10=-9.024617835750803, P11=-1.639400861152857, P12=-6.917873451924654, P13=-3.165646131995793, P14=2.2307825662519085, P15=0.3484753760787799, P16=11.51762222829602, P17=8.081883521503958, P18=11.135588485691258, P19=17.557271049195016, P20=11.137857693708161, P21=16.14732836455931, P22=7.512024722448528, P23=6.074090212321605, P24=1.066981439501378, P25=4.574146956954454, P26=1.7729163239991859, P27=2.7603678403569067, P28=2.74249970812737, P29=0.17986534604019025, P30=-1.9630709072796593, P31=2.974020970815817, P32=-1.0703604191367808, P33=1.6763328838386082, P34=-2.8061155759425076, P35=-1.3720185834699095, P36=0.8640740561434359, P37=2.5188131004787997, P38=-0.6356798773463327, P39=1.5254698860835039, P40=0.5984553397336696, P41=0.6101503842182749, P42=0.5970535883536532, P43=0.5651514644804753, P44=0.5982552300360295, P45=0.5969719004366049, P46=0.5672420846699617, P47=0.6151565107548143, P48=0.6696988302053082, P49=0.7408108549287942, P50=0.7132776129772364, P51=0.6494939868170017, P52=0.00013904606553627203, P53=1.8605217479451554e-05, P54=7.0132591916816025e-06, P55=1.3265883529576758e-06, P56=4.4657585887107405e-07, P57=6.402287138743915e-07, P58=7.235625925779912e-07, P59=3.978936813161486e-06, P60=0.00015759011851467815, P61=0.0021154287115914194, P62=0.015959569783210673, P63=0.043822490234039245, P64=0.04000706459021722, P65=0.040641209577786476, P66=0.05526208528837099, P67=0.07427993502772898, P68=0.026514679116689976, P69=0.00255771374668937, P70=0.0011200224656598592, P71=0.0004984978691395874, P72=0.0005029481185555284, P73=0.0016443954348247085, P74=0.004910400994333499, P75=0.009624682640681108, P76=0.011504682377533257, P77=0.03631880809272545, P78=0.07568681160934233, P79=0.10050917052872332, P80=0.07234046934099739, P81=0.04209839321570755, P82=0.03646317396997108, P83=0.03525631067416682, P84=0.017079484057932536, P85=0.005884026652178221, P86=0.005282491805088492, P87=0.01000028035945872, P88=0.027841326542735287, P89=0.024926107921779017, P90=0.023853261033810122, P91=0.03152275471578458, P92=0.07888266892560554, P93=0.026228995669541465, P94=0.004087543895057598, P95=0.0010078827894265845, P96=0.004835594494673077, P97=0.018431925735674957, P98=0.007645821762768892, P99=0.0020051900694268714, P100=0.01345146817465964, P101=0.03289590039630545, P102=0.003712039363363722, P103=0.000485326366030835, P104=0.0009303317038868379, P105=0.00026144086533044466, P106=0.0004981171066955787, P107=0.003423133298613689, P108=0.004200478317730214, P109=0.0004626085803776921, P110=0.00033483172797054453, P111=0.0004578998153808973, P112=0.00039515277115780776, P113=0.001609836349812503, P114=0.0013265940940200215, P115=0.0010151574008667023, P116=0.0002569909595531576, P117=0.00046799131817254773, P118=0.0014751506280752378, P119=0.0006240069830943725, P120=0.00028241672712790906, P121=0.0003479139474843713, P122=0.0005437957558889716, P123=0.00038517256468709453, P124=0.0004251212208968696, P125=0.00036170182637496116, P126=0.0001502404161933573, P127=0.0001446391379068543, P128=0.0004698265681820401, P129=0.0006840130267353877, P130=0.00021797240365540116, P131=0.0004357300698346661, P132=0.0002339446672279035, P133=0.00025636913064627006, P134=0.0002774928846406352, P135=0.0003247390108052511, P136=0.00014792065676732223, P137=9.058706783834203e-05, P138=3.3014872680217463e-05, P139=2.05051713674431e-05, P140=2.0920309573820507e-05, P141=3.0649849582899987e-05, P142=4.734251367997092e-05, P143=0.00011581632227879046, P144=0.00010102466841623453, P145=8.886329695629974e-05, P146=4.1339400338112255e-05, P147=4.813379214812382e-05, P148=4.0277125928228275e-05, P149=5.006682275618894e-05, P150=4.693460491516767e-05, P151=3.87331696449198e-05, P152=2.5142846697496502e-05, P153=1.564772080778101e-05, P154=4.87314027018332e-06, P155=2.868965622182984e-06, P156=9.368197009127373e-06, P157=1.1554528978910215e-05, P158=7.372228611238839e-06, P159=5.971629460773584e-06, P160=1.3271937436308999e-05, P161=1.835959702629708e-05, P162=1.4334205527296735e-05, P163=1.4857015125812063e-05, P164=7.763907565647494e-06, P165=6.726688907764975e-06, P166=7.354281578482572e-06, P167=8.367144406084128e-06, P168=9.51798961639437e-06, P169=8.355177790006505e-06, P170=6.309598800634359e-06, P171=4.134907168473146e-06, P172=5.941857528709974e-06, P173=7.95524749306073e-06, P174=1.4033518143445517e-05, P175=1.3496831909999927e-05, P176=1.2886315491007317e-05, P177=1.461191460210459e-05, P178=7.078967993788033e-06, P179=6.994327569877178e-07),Row( P0=-628.9192815863064, P1=67.7732469563005, P2=-18.224592194606313, P3=5.438618818383155, P4=-13.076827744868492, P5=-13.414546239195985, P6=-2.001272991384192, P7=4.010472234242672, P8=-1.579595167545502, P9=-9.129797591652393, P10=-3.103476476656961, P11=-2.591431123336575, P12=-8.446129820911146, P13=1.3541932726423993, P14=-0.20725117408943947, P15=-4.530839506211196, P16=3.801684699603597, P17=-1.919674829938444, P18=0.22524249794448803, P19=7.683452821381798, P20=2.5699924483683643, P21=11.648417095890963, P22=6.543397022725299, P23=5.517210457557062, P24=2.6294095733834033, P25=3.1726283798048938, P26=-0.15700554972019032, P27=1.2077554079212052, P28=1.852704900870855, P29=2.8352406978696942, P30=-2.8073963425233477, P31=1.6512372954399372, P32=-3.663311640746338, P33=1.878153976538122, P34=-1.7013421022129411, P35=1.4285411633788503, P36=0.6540009754921812, P37=-0.41337066441031756, P38=-2.706674496192676, P39=0.8947614717109901, P40=0.653362182160888, P41=0.6082841080192656, P42=0.7397967486166257, P43=0.7224807357098777, P44=0.6302503564228444, P45=0.6738520356150598, P46=0.6961671599096754, P47=0.7238645025495061, P48=0.7081712862825521, P49=0.6552398624901452, P50=0.6487165161972354, P51=0.7005381405213755, P52=3.649460008541871e-05, P53=3.5702344367311906e-05, P54=3.088802739718393e-05, P55=7.708850799879829e-06, P56=8.550762857485725e-05, P57=0.0008942482042097168, P58=0.0006994103316385054, P59=0.000466940420657745, P60=0.002081833955399489, P61=0.00632226705560692, P62=0.023770053222858297, P63=0.05746482822500433, P64=0.04966092010896235, P65=0.017704581768103182, P66=0.012668291712178323, P67=0.011111619178114634, P68=0.0026258464123647533, P69=0.0013218495434988205, P70=0.003051837433639223, P71=0.001978796277468113, P72=0.0011845086125709277, P73=0.00292824814056324, P74=0.007699299301110483, P75=0.015123069934856724, P76=0.022151001406133942, P77=0.028229277652008996, P78=0.020626873372572328, P79=0.01866455397354298, P80=0.016170068301064093, P81=0.027047729717773514, P82=0.05731222948684676, P83=0.02173000624892405, P84=0.006639695624994572, P85=0.011772245168582766, P86=0.022010625111530856, P87=0.037660147069791594, P88=0.07673183807715833, P89=0.035865115874802095, P90=0.0163377059440676, P91=0.021495802583672615, P92=0.035735863549555734, P93=0.023101635659250927, P94=0.031643102457984705, P95=0.04201932868377824, P96=0.024071572901973027, P97=0.012510209097655878, P98=0.004675761813304057, P99=0.0020992267152218223, P100=0.0023683792775370176, P101=0.005268856730559317, P102=0.0071665534717406, P103=0.005537569496982615, P104=0.001428502908011965, P105=0.0005145652924607294, P106=0.0007329340555537851, P107=0.0027591161446207115, P108=0.004748079288533761, P109=0.0027915928335984484, P110=0.0019923523188785, P111=0.0004651634380123515, P112=0.0012436696430394095, P113=0.0013847676462448463, P114=0.0009067556489371763, P115=0.0004900231311592917, P116=0.0003943872700480433, P117=0.0008875372212910674, P118=0.0004965060635588438, P119=0.00026444450843137704, P120=0.0004409655435568201, P121=0.000854695501737013, P122=0.0009556388920296772, P123=0.000458146800843942, P124=0.0007116563446726547, P125=0.001547713274647309, P126=0.0010543995704167754, P127=0.00026390089044157157, P128=0.0004382164160946177, P129=0.0003572374017185177, P130=0.00020461295365470616, P131=0.0004388406981032946, P132=0.00048068365696793175, P133=0.0003876256348483158, P134=0.000368762017376969, P135=0.00033126611701056946, P136=0.0004222548583142369, P137=0.00011188901290997921, P138=5.452003742076274e-05, P139=3.666931700503762e-05, P140=1.6222991740506054e-05, P141=2.1824016615525294e-05, P142=4.2099298573364916e-05, P143=6.828681585555862e-05, P144=0.00014340485013119103, P145=0.000120429337394398, P146=0.00012481637194902822, P147=0.00010282954453947799, P148=0.00012911147594429098, P149=0.0001874767108878561, P150=8.920495611239338e-05, P151=7.832106837160784e-05, P152=9.90702234898727e-05, P153=4.135615126205622e-05, P154=1.6496937626874125e-05, P155=2.7302356462511364e-05, P156=4.285134071238237e-05, P157=0.00010106754539383825, P158=0.0001181749019972652, P159=6.631449884383271e-05, P160=0.00010653757199087676, P161=0.00011135727706906097, P162=9.32135097990754e-05, P163=5.931086315374601e-05, P164=2.5250123682981734e-05, P165=1.1637595447031224e-05, P166=2.37119128994025e-05, P167=4.0513138768413146e-05, P168=5.124524652741753e-05, P169=3.748275670948868e-05, P170=1.75295611011922e-05, P171=6.008214931091056e-06, P172=7.564191687550834e-06, P173=1.3505249948308558e-05, P174=2.379362075412939e-05, P175=2.3275769409791664e-05, P176=1.9996453005888192e-05, P177=1.4684106651105557e-05, P178=6.884806916042887e-06, P179=8.609788935200873e-07),Row( P0=-511.25186771951013, P1=54.32985757690559, P2=-32.31255634237196, P3=0.12793619818138666, P4=-19.821510483041987, P5=-22.59551451607035, P6=1.927353691385157, P7=-4.379448814690067, P8=-3.9652763408435048, P9=-7.438004338013033, P10=-11.276892379453372, P11=0.6876765579967093, P12=-8.75792370713414, P13=2.803158147029292, P14=3.926304106033171, P15=4.106144445342037, P16=10.511507685690185, P17=13.286862980158016, P18=11.158715648846256, P19=15.448113989977314, P20=5.658275761510732, P21=9.792363845499791, P22=5.013567212320182, P23=1.5016370143712108, P24=2.362283035174192, P25=5.614001511049951, P26=2.668363687307123, P27=1.4450277320895029, P28=0.3945327906536392, P29=-1.0285013372534546, P30=0.5341002623060852, P31=5.041818049352105, P32=-0.013109546532339974, P33=1.740239333474161, P34=-1.4572237667253016, P35=2.1321847768647637, P36=4.115051857721462, P37=4.996962720648051, P38=-1.3769173858927657, P39=-1.3998638935271819, P40=0.6477834884785728, P41=0.5968239359523735, P42=0.6240354348739157, P43=0.6303032716449215, P44=0.5988866073708281, P45=0.6428236035707626, P46=0.6016130695086089, P47=0.5015345457410804, P48=0.5048410707719418, P49=0.5701079828128547, P50=0.6771933352857541, P51=0.7075872077096522, P52=0.02775247341219854, P53=0.002787959766701781, P54=0.00032676848483707236, P55=3.967605343263765e-05, P56=9.008936939777077e-05, P57=7.528338180629995e-05, P58=2.3735463007456562e-05, P59=0.0001492284678126582, P60=0.0056221515606336885, P61=0.03278595774333829, P62=0.0592257824039236, P63=0.06582805824374512, P64=0.049625537164508016, P65=0.20909080682072792, P66=0.33564175505267063, P67=0.2501915784271182, P68=0.1262135444787081, P69=0.064252839100192, P70=0.04669557655883108, P71=0.01869192451835107, P72=0.011776130019667978, P73=0.023437564141281, P74=0.01964447974051567, P75=0.01511732214831921, P76=0.018894743423066244, P77=0.06322453380572712, P78=0.17849929443207474, P79=0.2980086439272844, P80=0.3772621276151441, P81=0.3675493581527377, P82=0.2729147096438343, P83=0.2752713905974683, P84=0.2995099555919385, P85=0.20528796313504477, P86=0.14459515727176975, P87=0.15053908476979583, P88=0.12077066133735544, P89=0.08258436300461929, P90=0.08737544699962102, P91=0.13295132536295642, P92=0.5380779217208058, P93=0.41056503374337955, P94=0.2104621663988388, P95=0.138375219623363, P96=0.051384842197827296, P97=0.02822525063621015, P98=0.07042978489167462, P99=0.09460790262634342, P100=0.1052252530759537, P101=0.26800053778643423, P102=0.12971955902062723, P103=0.034218940326983215, P104=0.007684186398536991, P105=0.00612635298508623, P106=0.009946023358056886, P107=0.018146609167464797, P108=0.03149622531792339, P109=0.015222019130174458, P110=0.015320669078674685, P111=0.005746053981274152, P112=0.009830838312781977, P113=0.01596026358728503, P114=0.009992262155918422, P115=0.0070867561339937254, P116=0.010759418171313904, P117=0.002961298906151368, P118=0.016153999271258494, P119=0.02528275441600921, P120=0.02666355871315856, P121=0.01224610339694394, P122=0.011687772384233384, P123=0.01533310003080757, P124=0.020068074941816352, P125=0.01650264909342337, P126=0.013183679993651217, P127=0.01513374265628172, P128=0.01665365371784566, P129=0.012026253923087425, P130=0.005863634992993444, P131=0.005650866821664114, P132=0.013612661403806245, P133=0.009115440388380561, P134=0.00803170092802311, P135=0.01158080009275609, P136=0.010500042907921512, P137=0.007389617720514242, P138=0.0038401113647970734, P139=0.0021684649183504948, P140=0.001441135950831117, P141=0.002223618799645528, P142=0.0022121426156943716, P143=0.0029721102240240424, P144=0.003590193991471023, P145=0.0027491137810127845, P146=0.002161301183866956, P147=0.0014063646847565713, P148=0.001385385164307783, P149=0.0009828348726638183, P150=0.0007449968811466798, P151=0.00031345110663875134, P152=0.0002573363777680358, P153=0.000166678157259321, P154=7.143587964964204e-05, P155=0.00013570567766046877, P156=0.0002735614069216225, P157=0.0004483332180865808, P158=0.0004955546777415911, P159=0.0009263360330692774, P160=0.0021298496476949143, P161=0.0015738760171628565, P162=0.0018597710946511806, P163=0.0009143918277242911, P164=0.0003079614373078556, P165=0.0006402101321401252, P166=0.0008631437597649155, P167=0.001030711202563377, P168=0.0008639209213789411, P169=0.000835638664361469, P170=0.0005294531253258832, P171=0.0002619470750076305, P172=0.00017799910893852582, P173=0.00019491150525984474, P174=0.00024310697943882043, P175=0.00018314528558808121, P176=0.0001573515246908079, P177=0.000136523884250011, P178=7.944786575518296e-05, P179=9.814982469075702e-06),Row( P0=-664.9417231101434, P1=41.02992306492508, P2=-12.001639481500613, P3=-10.541012854739197, P4=-21.108211406516666, P5=-21.65796489424011, P6=-9.363033986516736, P7=-4.551938072656166, P8=-5.959658721152042, P9=-10.847898457313054, P10=-9.248457615520623, P11=-9.436039824807585, P12=-11.214177570743601, P13=-5.997819692695122, P14=-7.29079280852392, P15=-5.52028969619332, P16=-2.4334750926913724, P17=-0.5363548806679693, P18=2.4589556402791413, P19=6.950951856740709, P20=7.473111786976476, P21=12.830762893916614, P22=9.803571746772972, P23=8.902611046519706, P24=2.581184021348163, P25=7.488203403471547, P26=1.1869630845420487, P27=0.8970159596396374, P28=1.3093346743032024, P29=-0.07754525079347487, P30=-1.530514151961519, P31=1.273932257106836, P32=-1.527227561661591, P33=-0.3401154661060241, P34=-2.37083601650566, P35=-2.0875226237463322, P36=-0.27106210481888793, P37=1.9029972433649716, P38=-1.7770628456485114, P39=0.40278462907232015, P40=0.5822746918704059, P41=0.6310000080444854, P42=0.6279038428026382, P43=0.6130146494540472, P44=0.5850623990034665, P45=0.6823132638582212, P46=0.6942657348924012, P47=0.6565135573143449, P48=0.6618380562715538, P49=0.6561269621472592, P50=0.6454672155788335, P51=0.6065867368681015, P52=4.855343157449354e-06, P53=1.3211838113588144e-06, P54=8.313961917760313e-07, P55=6.544795473084943e-07, P56=6.436802742594912e-07, P57=1.295065113873812e-06, P58=2.1630981062169673e-06, P59=1.3326847099901562e-05, P60=0.00040063492710479314, P61=0.004957573880116757, P62=0.03590468011116276, P63=0.05329771719076683, P64=0.03779291726986843, P65=0.02766674833409205, P66=0.019257346840852042, P67=0.004168655084399388, P68=0.00035446597865450595, P69=0.0001215326087440609, P70=0.00015963544797347677, P71=0.00029051314425245184, P72=0.0019109810388912365, P73=0.011069171069102648, P74=0.015087733161787655, P75=0.008962180920439138, P76=0.01960908625611278, P77=0.05392165064215586, P78=0.055087400561121574, P79=0.051918346056304895, P80=0.0547091630792815, P81=0.08535748891853073, P82=0.07446569666216822, P83=0.014425055260599864, P84=0.015120294982042573, P85=0.017493975266579333, P86=0.009218931702325999, P87=0.006931077532136976, P88=0.0218200066234339, P89=0.02489263750014946, P90=0.019594738145554014, P91=0.014481164247346159, P92=0.025676578184908302, P93=0.035035946081633296, P94=0.03343911034726286, P95=0.004955658538870473, P96=0.0006536429275662849, P97=0.0019417700230599329, P98=0.0014041598713110245, P99=0.0033308007137998964, P100=0.0036750667912904714, P101=0.0015428474901509933, P102=0.0019028475134616899, P103=0.001499889295369672, P104=0.0002899454667059449, P105=0.0008967037433414767, P106=0.0010605946484624701, P107=0.000491636071186436, P108=0.0004259518853782639, P109=0.0003279146617113046, P110=0.00040642707120084454, P111=0.000369352083708815, P112=0.0005369735728238706, P113=0.000789952405368234, P114=0.0004663035442455501, P115=0.0005042583330364495, P116=0.00010476816716517895, P117=5.883337907664691e-05, P118=0.00018519381528129655, P119=0.0001738462868159631, P120=5.851284337343375e-05, P121=5.5610998045516875e-05, P122=0.0001705689342787835, P123=0.0002745316323255597, P124=0.00013290143510702522, P125=0.000204860791259899, P126=0.00010859991855028272, P127=8.273816271443354e-05, P128=9.3358984148213e-05, P129=0.0002215849949223024, P130=9.892061285448983e-05, P131=0.00023503482682610618, P132=0.00017485570503074144, P133=0.00011024690401075639, P134=0.0001965906543968016, P135=0.00019528050206902724, P136=0.00010140131746480458, P137=6.182184522686223e-05, P138=2.479040479936127e-05, P139=1.7335185656655098e-05, P140=2.2055990150380108e-05, P141=3.5133837876778644e-05, P142=4.478332339186324e-05, P143=9.85423701692742e-05, P144=3.722018692650692e-05, P145=2.517000286588506e-05, P146=1.3487658308805117e-05, P147=1.493869388844716e-05, P148=2.7625640794034768e-05, P149=5.1520180501014154e-05, P150=6.886567225330403e-05, P151=7.464522873375827e-05, P152=5.833825422937342e-05, P153=7.846659632625644e-05, P154=7.707560764776573e-05, P155=6.0130501687133394e-05, P156=7.324106460752321e-05, P157=5.6684247081325094e-05, P158=7.757690930229835e-05, P159=0.00010651201275857807, P160=0.0001167556029719561, P161=0.0001105452993661121, P162=0.00012995756078328959, P163=6.807763766815132e-05, P164=5.096729442312167e-05, P165=4.129909505499092e-05, P166=2.8849464653527485e-05, P167=2.68976149133757e-05, P168=4.754083797501198e-05, P169=5.4410690108777716e-05, P170=5.2893578177005564e-05, P171=4.4718119690023444e-05, P172=2.4083256758599848e-05, P173=1.8403512131733136e-05, P174=3.1851916808987855e-05, P175=7.533110881504134e-05, P176=9.515007158838645e-05, P177=7.276149306595669e-05, P178=2.988480349449761e-05, P179=3.2829838856891126e-06),Row( P0=-457.86253152119616, P1=42.48711834273834, P2=-21.20454606289988, P3=1.9534705934449301, P4=-19.88644318828005, P5=-16.469388479998678, P6=1.3265284013974845, P7=1.9717628291775338, P8=-4.938226419023039, P9=-2.283623967388685, P10=-4.80527769128066, P11=0.37945753623519646, P12=-3.227786540414409, P13=4.905754670907144, P14=9.145215284170678, P15=10.054204398086286, P16=16.04560179678619, P17=10.087161557340739, P18=7.979853927491541, P19=7.0828670345901195, P20=-1.1562197466938462, P21=4.425292337712186, P22=0.9623675041232902, P23=2.6080536962382905, P24=-1.9777100000087962, P25=2.8617417189383496, P26=0.4447094945805836, P27=2.8180718914105185, P28=2.607162168297138, P29=3.92047482452875, P30=2.178402675265122, P31=6.219843088316194, P32=0.18605279440801553, P33=3.2744283171874935, P34=-0.5340280495314971, P35=1.649257522896707, P36=1.8545046701792167, P37=3.5458234156906383, P38=0.129111494869835, P39=2.7185739849712762, P40=0.7063073911780502, P41=0.7090744243215168, P42=0.7079222244353656, P43=0.6891857080722151, P44=0.6406051738572361, P45=0.6180332834525698, P46=0.5927226441259926, P47=0.575936496165694, P48=0.5731699992469079, P49=0.6179729898873115, P50=0.6768051920961587, P51=0.727147028824726, P52=0.06762722950162084, P53=0.006660706074817588, P54=0.0003834007022273947, P55=0.0002494890767429935, P56=0.0006700077604853635, P57=0.0032604080502479475, P58=0.005956982369014967, P59=0.0077801961165175175, P60=0.008664493169179034, P61=0.012963404472975013, P62=0.029721531870779958, P63=0.056365699154737284, P64=0.06250482369715711, P65=0.22212822407001828, P66=0.6423911057667752, P67=0.6693855706350406, P68=0.526069928256705, P69=0.6091830395710122, P70=0.667666607335281, P71=0.2206579618749932, P72=0.024792139291158297, P73=0.018456065298380984, P74=0.014366802165136994, P75=0.00969023055177142, P76=0.013492203756556537, P77=0.04536682887311939, P78=0.07116323414082115, P79=0.0899812130670701, P80=0.1345057687369747, P81=0.2983532789484582, P82=0.5248203230145935, P83=1.371898845883083, P84=2.007212282883839, P85=0.9276938519422953, P86=0.40191193167923367, P87=0.3715066927330197, P88=0.5862231791536321, P89=0.22991073650742525, P90=0.18564423931694402, P91=0.15777501658529797, P92=0.185088295421536, P93=0.28819223621179896, P94=0.3469923423389392, P95=0.2502950382351752, P96=0.26852471347527646, P97=0.22514404680216643, P98=0.34109662538202684, P99=0.25485136480981196, P100=0.08446725157883966, P101=0.1064417056990237, P102=0.2182436057067141, P103=0.07300238340639802, P104=0.0341017492105277, P105=0.028247456846551895, P106=0.09412969618832427, P107=0.09687089528794882, P108=0.025960676002676073, P109=0.0178222010795404, P110=0.019297161666981654, P111=0.07953492267342165, P112=0.03816520538153497, P113=0.008778613417121646, P114=0.012701997684242882, P115=0.012032979443842443, P116=0.006084963819030237, P117=0.005584027262606059, P118=0.005595863204704415, P119=0.009891279684663235, P120=0.016094022649296606, P121=0.009008635146803947, P122=0.004787123009706469, P123=0.008707692151397075, P124=0.035483247012639066, P125=0.024198243397772087, P126=0.018339749669224196, P127=0.02185644632878201, P128=0.025800142455147786, P129=0.029518075504451517, P130=0.017951166666388088, P131=0.021293051234850903, P132=0.018285761961964592, P133=0.030931953383543348, P134=0.033880505765162626, P135=0.034583097745538784, P136=0.019703384464396758, P137=0.012729283682689871, P138=0.005578899998054506, P139=0.003408072817683388, P140=0.0040118504818556526, P141=0.002771359732866111, P142=0.003770171553911805, P143=0.004645752255902163, P144=0.0028567954408911782, P145=0.0030037466699910174, P146=0.0023663822927020156, P147=0.0036081760523249823, P148=0.004156793007444623, P149=0.002418222550389926, P150=0.005019584837092668, P151=0.0033368330312618976, P152=0.0019459145840575788, P153=0.0006011393540267885, P154=0.0005634226193998667, P155=0.0004973915783766338, P156=0.00035904130335700916, P157=0.00021164299773982555, P158=0.00033256844092445097, P159=0.0004240234107058668, P160=0.00034647224285461897, P161=0.0004287524334644252, P162=0.0004580165170667697, P163=0.00037790253599620375, P164=0.00040165190219240937, P165=0.00028388817656401535, P166=0.0002221505345752483, P167=0.00015665702904039304, P168=0.00015519375086337576, P169=0.0001397597864589644, P170=0.0001319529501146601, P171=0.00014864739138740248, P172=0.0001509968615851774, P173=0.0001956598779480346, P174=0.00026100438262815546, P175=0.00022919662009879518, P176=0.00022569942228393617, P177=0.00018687493154300152, P178=8.197064833321305e-05, P179=9.212628433852245e-06),Row( P0=-647.3224428215096, P1=68.67044555985251, P2=-4.924128077072658, P3=11.81509238524122, P4=-11.376257122869651, P5=-15.756075893406608, P6=4.078971791627073, P7=9.32849041504694, P8=-3.542899889990937, P9=-8.182772239836899, P10=4.573235652998505, P11=-4.102480383171143, P12=0.637622356384524, P13=-0.0909307765824294, P14=-0.08312136674977831, P15=-2.6389149808008305, P16=0.8213139934038198, P17=-2.048499759045893, P18=-6.3631291205760325, P19=-2.8859403641883348, P20=-5.795912174401883, P21=2.03893597948897, P22=-1.6152559029496751, P23=-0.7254356261200423, P24=-2.8804968596159113, P25=2.449640110287558, P26=-6.544605993529648, P27=1.1301205749560812, P28=-1.9415227939036621, P29=2.559491384721531, P30=1.260807102894432, P31=8.057313617274897, P32=2.503158120798403, P33=7.881322205717608, P34=-0.6508869651466087, P35=5.0090290467819605, P36=3.225608918117747, P37=5.6817709654628885, P38=1.3437941567325176, P39=2.2731318455351275, P40=0.7636107680098292, P41=0.7110505873767748, P42=0.7009122083382302, P43=0.6699769511074858, P44=0.620778503098253, P45=0.6531243691226435, P46=0.6998241356481384, P47=0.6403183528436849, P48=0.6422504589236703, P49=0.6542210959633108, P50=0.6453059003652207, P51=0.6884891916393947, P52=0.0006101582937785341, P53=3.478953356774055e-05, P54=2.6106878393976954e-05, P55=7.867447442714639e-06, P56=0.0002949111532822074, P57=0.005195094335123229, P58=0.01026415139247844, P59=0.01613117597727611, P60=0.008447097477784828, P61=0.002333871199463515, P62=0.00413658931400979, P63=0.013252180397581359, P64=0.00917233827230916, P65=0.0014115681703727777, P66=0.0025161984083656727, P67=0.0063539514131558, P68=0.003458962264898752, P69=0.0019064546760097548, P70=0.0014128547808966824, P71=0.000934568766775209, P72=0.0005138321413787167, P73=0.001372054886225117, P74=0.002743080764840528, P75=0.005539241697387499, P76=0.007421157675669158, P77=0.0052844579425131094, P78=0.0029914718038440437, P79=0.0012509976051511907, P80=0.0004446139937895254, P81=0.0004649934670541262, P82=0.0008634689181254485, P83=0.002473103779887414, P84=0.003013368104161679, P85=0.0020885999577430103, P86=0.001170037333433598, P87=0.0018372911104715059, P88=0.003356342089435273, P89=0.0013894456935801725, P90=0.0008657984106486399, P91=0.0004835637207456032, P92=0.00021223664717418382, P93=0.00042317183119556427, P94=0.0011672428653374544, P95=0.0009139190746794634, P96=0.0006122878401399945, P97=0.00022693731037061007, P98=0.00032829861860282324, P99=0.0006244617339975688, P100=0.0005822754733955877, P101=0.0004981914956296326, P102=0.00023454457216426805, P103=0.00018020264138572447, P104=6.357029238941918e-05, P105=2.3698836421355682e-05, P106=5.3424521771294196e-05, P107=8.99912929618044e-05, P108=0.0001975244628135311, P109=0.00010177743724652439, P110=3.178076467687033e-05, P111=8.348052474938303e-06, P112=1.8492634442128525e-05, P113=4.41147796068381e-05, P114=3.640591768170897e-05, P115=1.707096371167098e-05, P116=1.124649732794314e-05, P117=1.397320080653254e-05, P118=2.917159625985573e-05, P119=3.509791769720224e-05, P120=4.671174959865009e-05, P121=4.60903307203137e-05, P122=4.730496099383714e-05, P123=5.477924599074563e-05, P124=0.00010441191873751327, P125=0.00011808271320790637, P126=0.0001128298576978876, P127=0.0001805933602384919, P128=0.00017792037293060472, P129=5.742208460252034e-05, P130=3.6224589676593894e-05, P131=4.324319493538197e-05, P132=7.288410435436253e-05, P133=7.461904928565944e-05, P134=8.290710820119876e-05, P135=8.949988268477575e-05, P136=6.236417672471028e-05, P137=3.0838270305362754e-05, P138=1.3940462203181956e-05, P139=2.113972680474702e-05, P140=1.0459658534050281e-05, P141=5.710199834656574e-06, P142=3.837309136344633e-06, P143=5.79843820364213e-06, P144=7.433140092451747e-06, P145=8.120903320032099e-06, P146=7.656845911544056e-06, P147=3.6760552776342067e-06, P148=3.836324407151512e-06, P149=3.0399923911981538e-06, P150=4.2805999980868644e-06, P151=5.157713636994707e-06, P152=7.884504430528639e-06, P153=1.2241998735314484e-05, P154=8.040875462673812e-06, P155=5.93214188345898e-06, P156=5.8779337473729e-06, P157=7.37895131525423e-06, P158=3.9421085580867165e-06, P159=4.1856339969684546e-06, P160=6.4222508247216e-06, P161=6.404287176207544e-06, P162=5.7213269151701165e-06, P163=5.169285069970972e-06, P164=6.517901742526403e-06, P165=5.29941261265426e-06, P166=5.246643416427551e-06, P167=3.6076631846670122e-06, P168=4.183511648008344e-06, P169=3.386807538573514e-06, P170=1.71559552452249e-06, P171=1.7775116321135272e-06, P172=2.5002838627538683e-06, P173=2.6492827413334736e-06, P174=4.983997319648122e-06, P175=7.788446072964657e-06, P176=8.412929397217386e-06, P177=6.233412239629694e-06, P178=2.4846392035570787e-06, P179=2.572700866665036e-07),Row( P0=-704.4882838398612, P1=80.61450429880125, P2=-4.058379675360513, P3=4.545443854527945, P4=-4.503466298196802, P5=-11.861295453696412, P6=2.169782102099262, P7=2.7446318816096102, P8=0.06349405553289884, P9=-6.2567183540681945, P10=0.788844277934341, P11=-4.162873569473709, P12=-4.880732731002759, P13=-0.24409768905471765, P14=-2.964790561022312, P15=-4.164694677806718, P16=2.4405304807002404, P17=-2.540013640302259, P18=-2.368794792265942, P19=-1.9862439907548284, P20=-7.36554764231425, P21=-1.1639581478675542, P22=-3.233269029336633, P23=-2.5967127353058643, P24=-5.204929201880461, P25=1.2592565004041802, P26=-1.5696193926377187, P27=1.8407656865071487, P28=1.815765498509865, P29=0.6372626871946387, P30=-1.773432374679891, P31=2.079923513038821, P32=-2.4398854354090806, P33=0.8563337472910341, P34=-4.323798983208048, P35=1.6303782323301903, P36=0.5742981577878041, P37=4.838104553300652, P38=-0.37470714892954543, P39=0.4765722552136475, P40=0.6453861083721859, P41=0.6813384259467012, P42=0.6948837769312457, P43=0.6640230027452304, P44=0.6528215690822824, P45=0.6414465854229564, P46=0.6329675310107706, P47=0.5672455016012405, P48=0.6315366628520049, P49=0.6826670578723906, P50=0.6610319691047553, P51=0.6333664256797196, P52=2.376015773687195e-05, P53=1.3096695275089034e-05, P54=1.1053754718541786e-05, P55=1.86530189724455e-05, P56=0.0016836382482704252, P57=0.008280980086317275, P58=0.0049000020010458905, P59=0.0017206631816830524, P60=0.003792713724206509, P61=0.005445174694219383, P62=0.002871913443380465, P63=0.0012254619615611638, P64=0.0009470870231664447, P65=0.0012870063938004478, P66=0.0013186055933479238, P67=0.0009539865893866341, P68=0.0012300965577663817, P69=0.004159663570382274, P70=0.007588088627263192, P71=0.005103974499209515, P72=0.003256533963478494, P73=0.0012478024734307474, P74=0.0005327487966613685, P75=0.0004548186442594142, P76=0.0006117898036243416, P77=0.0022396051289809526, P78=0.006057360501788153, P79=0.010013965814657829, P80=0.006516072315640728, P81=0.009982950215697365, P82=0.01504707283244682, P83=0.0042483225670000065, P84=0.0014729502015163735, P85=0.0026049182161313896, P86=0.0038297124781459108, P87=0.002947120436885823, P88=0.0038400352833816522, P89=0.002236343735542356, P90=0.0019892535922464345, P91=0.0015989245587768148, P92=0.00046113410281007906, P93=0.00165186668841535, P94=0.011308153665760661, P95=0.00930063806647577, P96=0.0018205016339136748, P97=0.0004716940885170785, P98=0.0012810452256229888, P99=0.0026540775499131514, P100=0.005735558746039555, P101=0.004655311783775057, P102=0.0009377102425306768, P103=3.769368168344841e-05, P104=5.924984656773364e-05, P105=0.00010134226145025388, P106=8.211425001495115e-05, P107=0.00021659129738638237, P108=0.0018613715963184533, P109=0.002149791926346712, P110=0.0007171220336199902, P111=8.031829110490763e-05, P112=0.00031747990122831665, P113=0.0005709846111010728, P114=0.00024694727508751594, P115=0.00019211054222705915, P116=7.900376455616927e-05, P117=5.681242058925089e-05, P118=8.074955097660537e-05, P119=6.939507060690018e-05, P120=0.00010346181478964293, P121=0.0001156346813279486, P122=0.00011375946974968451, P123=0.00010106335399524195, P124=0.0001145504818111189, P125=9.197412616103181e-05, P126=5.3509987915164936e-05, P127=9.988640479470296e-05, P128=6.118250397773048e-05, P129=5.3247466648630816e-05, P130=4.58514928727593e-05, P131=7.648699753396483e-05, P132=8.325622786758144e-05, P133=0.00013460539588243457, P134=0.00010493088378168698, P135=4.069938687679422e-05, P136=1.984217088705782e-05, P137=1.2655904579123776e-05, P138=8.22568804092412e-06, P139=4.230706488634951e-06, P140=1.8911573493141355e-06, P141=2.145282029491677e-06, P142=1.5963655562875618e-06, P143=4.694612281617066e-06, P144=7.072883260332878e-06, P145=1.0309238003314952e-05, P146=7.979545347808811e-06, P147=4.295930744452192e-06, P148=5.629483491804469e-06, P149=8.412098442944385e-06, P150=7.420266512028491e-06, P151=8.715962836343605e-06, P152=1.1404168529143006e-05, P153=6.014222413727675e-06, P154=7.066668288195768e-06, P155=3.668965671544158e-06, P156=2.482624775233825e-06, P157=2.0355569272456054e-06, P158=1.7051516610489976e-06, P159=2.4423671541444447e-06, P160=6.3546743173627955e-06, P161=7.658209049395392e-06, P162=8.45288293177928e-06, P163=6.195387930981662e-06, P164=2.9614584806889187e-06, P165=7.920695074074929e-07, P166=9.169761488975803e-07, P167=1.8266507792279604e-06, P168=3.612261911003856e-06, P169=3.7539314014142407e-06, P170=2.941492321685035e-06, P171=1.1575812673497827e-06, P172=4.909293180959899e-07, P173=1.3363789686016985e-06, P174=2.8819033326246374e-06, P175=4.336644951105426e-06, P176=5.626918468818195e-06, P177=4.49320821067687e-06, P178=1.4695497693477967e-06, P179=1.3826683121036688e-07),Row( P0=-734.0251282547265, P1=71.5061097504436, P2=5.057950906899612, P3=2.1487251944254857, P4=-5.0932462552883315, P5=-7.2576807289311205, P6=1.7576225563058256, P7=6.169981778381644, P8=-1.462670867991908, P9=-4.0251439042042305, P10=-1.3619559137485016, P11=-5.177677564444387, P12=-6.218749864069313, P13=-4.423785228001887, P14=-3.0334212940088685, P15=-3.1232881647790753, P16=-0.5688001561614182, P17=-4.472030325116885, P18=-3.479023347116504, P19=-1.6720257317808331, P20=-6.304350329691176, P21=-2.205232154944168, P22=-0.5206928998712299, P23=-0.6334702592788007, P24=-4.939641051223036, P25=-1.27716485992975, P26=-2.3726373567002645, P27=-2.106897992540428, P28=-0.4520910759708768, P29=-1.2074608572412986, P30=0.47006241484681666, P31=4.801907086853778, P32=1.2926510112266307, P33=3.125663294998518, P34=-0.38922141733829674, P35=1.0634610885024702, P36=1.3113884753945828, P37=4.024683885826031, P38=0.6482386863643211, P39=2.546747748590989, P40=0.7578039192925203, P41=0.7471765365352595, P42=0.7388894637065484, P43=0.6815358908076893, P44=0.6600350261825219, P45=0.7114608164260177, P46=0.7011875727263278, P47=0.7242002031868816, P48=0.7332885458904977, P49=0.7371634381081854, P50=0.7080129009421132, P51=0.7016045930944782, P52=6.5315374341597344e-06, P53=1.1520196816854464e-05, P54=1.753742411613535e-05, P55=1.782307870658373e-05, P56=0.0007823581233272616, P57=0.004489267530443229, P58=0.0045621836986934, P59=0.005574069000977003, P60=0.0028592231971680867, P61=0.0008985910767420001, P62=0.002850315565880037, P63=0.0050709357696185065, P64=0.0015131519904875497, P65=0.0012617230480631723, P66=0.0028905303403786936, P67=0.0059713804699813725, P68=0.004748933886353168, P69=0.0014665792691717751, P70=0.0012553511136136758, P71=0.0008239199871944452, P72=0.0005347628289725652, P73=0.0011571128945684816, P74=0.00264044978981405, P75=0.004358355420911725, P76=0.0037067075795617905, P77=0.0017613834614907614, P78=0.002371630637217242, P79=0.0030992414478670533, P80=0.002532683749097693, P81=0.004609784741897244, P82=0.008420319687611826, P83=0.00999861227162812, P84=0.011202753110854393, P85=0.006222941983540802, P86=0.0039701498493330024, P87=0.004515370939141516, P88=0.002927440952453542, P89=0.00094923615863846, P90=0.001381600100541236, P91=0.0036958506581291567, P92=0.003475026674766633, P93=0.0014295341949706812, P94=0.0008316892115646714, P95=0.001154984180415049, P96=0.0009431070118059769, P97=0.0010639827117217846, P98=0.0006089902849130284, P99=0.00092435476045986, P100=0.0004447193316836785, P101=0.0003694899687512996, P102=0.00018518687480717028, P103=9.954024370361726e-05, P104=8.42088214036537e-05, P105=0.0001086632208102915, P106=0.00016861561498048963, P107=0.0003010751143980153, P108=0.0004888707986790926, P109=0.0003177487892291249, P110=0.0001527750931687936, P111=4.742184041223853e-05, P112=9.205462160484431e-05, P113=0.00013353749227084725, P114=6.610088844029469e-05, P115=3.6390936509635285e-05, P116=3.6545662279005356e-05, P117=2.7797992075836515e-05, P118=0.00010416111982754254, P119=7.517892787604645e-05, P120=6.59869253340677e-05, P121=9.296928080472036e-05, P122=4.919691309303897e-05, P123=0.00010013007624713216, P124=8.349110086859683e-05, P125=7.892325588637852e-05, P126=4.780951209187039e-05, P127=8.109426480382064e-05, P128=0.00011351999538437267, P129=8.217987279649e-05, P130=6.041457137560165e-05, P131=8.408585313222686e-05, P132=9.374709143029554e-05, P133=9.07907146307423e-05, P134=8.096668092687357e-05, P135=5.337647102161894e-05, P136=1.549959641646967e-05, P137=6.21305279471537e-06, P138=5.167605367881512e-06, P139=3.7077666215756604e-06, P140=4.53715048835526e-06, P141=1.3414898686848058e-05, P142=9.431508677950414e-06, P143=2.7058666957087812e-05, P144=1.8147801659123013e-05, P145=7.170553037258355e-06, P146=7.10875833220921e-06, P147=8.05190586735117e-06, P148=7.304427789305718e-06, P149=5.457898778674257e-06, P150=1.1168945730828142e-05, P151=2.6326508997288855e-05, P152=2.647229068016132e-05, P153=3.301880939674676e-05, P154=2.579518511368634e-05, P155=2.273080565720061e-05, P156=1.1313523499614226e-05, P157=1.0677126729245305e-05, P158=2.4729535712747275e-05, P159=3.558661504983559e-05, P160=5.1599292102576046e-05, P161=5.262401597376576e-05, P162=8.150034857152986e-05, P163=3.9801585705611284e-05, P164=3.544298025844694e-05, P165=3.125619662234505e-05, P166=1.4839902897284888e-05, P167=5.504681094764129e-06, P168=2.0080159164149255e-05, P169=2.2660441545341763e-05, P170=2.2926230091041723e-05, P171=2.8573729721150387e-05, P172=1.2180621432593156e-05, P173=5.794114402388673e-06, P174=6.267656611454397e-06, P175=1.4524404677335863e-05, P176=1.7946802517794698e-05, P177=1.5335993499254993e-05, P178=1.0625130806064074e-05, P179=1.442638952732293e-06),Row( P0=-699.6581902880248, P1=91.86598496491878, P2=5.88791731010688, P3=13.101097114587155, P4=2.3362513301469336, P5=-4.480622943639977, P6=1.2340815062278103, P7=8.855477872036701, P8=5.427571839272999, P9=-2.2955867301927086, P10=2.2908647221758383, P11=0.5431979188691263, P12=-1.194879905949152, P13=2.224580716926444, P14=2.782535830360856, P15=-0.681495427024122, P16=4.612671461829867, P17=4.086069922061982, P18=-2.591876760595403, P19=2.824179604154612, P20=-3.7020064501619756, P21=1.0886968337816045, P22=0.5709814330498841, P23=-0.0631347733908925, P24=-1.260874860160786, P25=2.7897855371742706, P26=-2.35980541559243, P27=3.7907552575295096, P28=-0.3567236957224206, P29=3.838867531951498, P30=-2.8366731936505984, P31=6.1761338569994715, P32=3.819648638409991, P33=5.7198500036413815, P34=0.8373242304915487, P35=3.1760776548427962, P36=1.1259614353780523, P37=3.5139904915999565, P38=0.7567323180411794, P39=0.8248831128372702, P40=0.769366520259166, P41=0.6954782915243947, P42=0.7202223366382401, P43=0.6683666053094623, P44=0.6759163721883392, P45=0.7048730541634515, P46=0.724397777419467, P47=0.7041053339959507, P48=0.7085267976904247, P49=0.690294594433397, P50=0.6857817940702985, P51=0.7629306162351547, P52=0.00016552024752346126, P53=0.0005278025265691533, P54=0.0002748021756584786, P55=1.2706193023471257e-05, P56=0.0008101120581986136, P57=0.005774785864414415, P58=0.004131945136157897, P59=0.004243370876863114, P60=0.004052839237318004, P61=0.005427756315294831, P62=0.004226051992779231, P63=0.0013243870556126443, P64=0.0007941787100428125, P65=0.0009935590278865915, P66=0.002731875640743205, P67=0.004026462069239516, P68=0.005525903805227668, P69=0.003932040914514564, P70=0.001581641901126108, P71=0.0012767367042764762, P72=0.0013620193537026174, P73=0.0015012707554684019, P74=0.0015524458795561375, P75=0.0018132970611016363, P76=0.002059089178666055, P77=0.004549028353990316, P78=0.004579288631048977, P79=0.0029153040373759652, P80=0.001679591008564816, P81=0.0026574637572302586, P82=0.00500431820994244, P83=0.006118190523976432, P84=0.0035727435615016154, P85=0.0027777101555880607, P86=0.0039013436267896014, P87=0.002907784443926635, P88=0.0035810667554679676, P89=0.0033507377641981504, P90=0.002796304896684374, P91=0.005555349105343743, P92=0.003515305501124666, P93=0.0034434132274804136, P94=0.0046308811126597, P95=0.0011411406393357123, P96=0.0004867010716143578, P97=0.002300431904698968, P98=0.004890621825814653, P99=0.00197742612271927, P100=0.00014137413257053752, P101=0.00021384658080681807, P102=0.0009069866902540237, P103=0.0007017866760224444, P104=0.0003090438474434121, P105=9.652083717603324e-05, P106=0.00011769918383346722, P107=0.00032547946631740504, P108=0.0013442261978700775, P109=0.0011565758771522167, P110=6.235142078057609e-05, P111=0.00020566867063067466, P112=0.0006284411902259593, P113=0.0009603889340293236, P114=0.0005542018064981462, P115=0.00037252432269754674, P116=0.00014145152943004417, P117=0.0003948970775261901, P118=0.00022032124134798884, P119=0.00015296402344072753, P120=0.0004102636993888972, P121=0.0003118796853257338, P122=0.00011471242584252611, P123=0.0003658937651220243, P124=0.00016281752310998756, P125=0.00018676261545127218, P126=0.00017542750238073762, P127=5.765787014340959e-05, P128=0.00013739569110724373, P129=0.00011461854540132617, P130=8.744703497838957e-05, P131=0.000113519526951732, P132=0.0001219755070505361, P133=9.299541986816559e-05, P134=5.892308142753237e-05, P135=5.5932586808669716e-05, P136=1.6866305477440697e-05, P137=6.535620783300366e-06, P138=3.6800475617719604e-06, P139=2.764781119509603e-06, P140=2.713238931269509e-06, P141=5.199913685210109e-06, P142=4.6216086209410285e-06, P143=9.23274885946578e-06, P144=1.0650741865945475e-05, P145=1.6488193980658117e-05, P146=1.5459555362476583e-05, P147=1.014720014079027e-05, P148=1.2601312296179158e-05, P149=8.846065202440408e-06, P150=5.197463682489032e-06, P151=4.661737774280632e-06, P152=3.6368609336019473e-06, P153=4.0672592579571065e-06, P154=1.2791913449992676e-05, P155=2.2227781204676898e-05, P156=3.0356887079303953e-05, P157=6.944650184187921e-05, P158=0.00012688026776748975, P159=0.00020491379551179065, P160=6.632553582587111e-05, P161=2.5560572898645183e-05, P162=1.4329569165465707e-05, P163=4.586671193365339e-06, P164=6.33259416736729e-06, P165=1.013837014461598e-05, P166=2.3745782612922537e-05, P167=2.3610868929987285e-05, P168=1.660449239471329e-05, P169=6.2148223137666615e-06, P170=2.951874342219182e-06, P171=2.848429632787928e-06, P172=1.3103188140929115e-05, P173=1.4620369151171268e-05, P174=8.360826748056665e-06, P175=8.64063233437478e-06, P176=8.285881391239481e-06, P177=8.353885799367507e-06, P178=3.253456419829642e-06, P179=2.469754876822503e-07),Row( P0=-721.0480374963918, P1=82.74251298827272, P2=-11.363423646999733, P3=3.7100268363502424, P4=-5.836605386717952, P5=-11.941503946326673, P6=-0.025276492717213578, P7=1.2209827281607684, P8=-1.8978051960966, P9=-8.8979057979889, P10=-1.4105475946846393, P11=-5.95707398558437, P12=-6.4177008995585485, P13=-1.9069288819820718, P14=-3.1777549982315216, P15=-6.5486928095572985, P16=0.32138212718129444, P17=-4.500535877135153, P18=-3.995575913721511, P19=-1.4302579724147564, P20=-8.117034976062515, P21=-1.0415785783385587, P22=-4.338559235377485, P23=-4.591188194445789, P24=-5.297980620422815, P25=-2.1837759736477653, P26=-4.203977488939875, P27=-1.9400986332172154, P28=-2.4223653860539156, P29=-2.3355623440000923, P30=-3.426677290621992, P31=2.0502382666621726, P32=-1.9638091915306357, P33=3.182604909296373, P34=-2.1428229771969227, P35=4.2298393401212016, P36=2.1103822382614483, P37=7.545716530934217, P38=1.5806415015411606, P39=2.1225092724035672, P40=0.7200600010333226, P41=0.7504327895274089, P42=0.7404002692178028, P43=0.6745594402130528, P44=0.6810418638231089, P45=0.7426958069155468, P46=0.763403603615162, P47=0.714474275657793, P48=0.7215597088685968, P49=0.7409955248140823, P50=0.7674872335005859, P51=0.7415356294450388, P52=1.3488003397777286e-05, P53=6.963840784193627e-06, P54=6.088196766012477e-06, P55=7.462048839258907e-06, P56=0.0011868171893954962, P57=0.004970342045426478, P58=0.00488609902437242, P59=0.005387629226962463, P60=0.0016914694448597221, P61=0.00023585550743138493, P62=0.0010218609550874756, P63=0.0015308747013013423, P64=0.0007415650447322385, P65=0.001658090770802743, P66=0.0035642009087386095, P67=0.004492912087949727, P68=0.0034491153875088184, P69=0.0012774263016442741, P70=0.0007906731451878067, P71=0.0006398643527254829, P72=0.0008370337524550736, P73=0.00204529992294496, P74=0.0016310985857686008, P75=0.001291988721737436, P76=0.003237526745159141, P77=0.0034509491419535904, P78=0.0015126765450504966, P79=0.0023180512465877, P80=0.003588926998631776, P81=0.0025480997356016567, P82=0.0015121131459814962, P83=0.0016480452434613866, P84=0.0023537771429993164, P85=0.004325676676443216, P86=0.0031111688523615413, P87=0.0018919195013921255, P88=0.0023587386981041326, P89=0.0010299742582224212, P90=0.0016651525849074693, P91=0.002046528776437314, P92=0.0027722565333517307, P93=0.0038539896084031485, P94=0.005386430033803643, P95=0.0030327257217924015, P96=0.0010051846262784734, P97=0.0021230274929432743, P98=0.00556801303688338, P99=0.004234158346838863, P100=0.0007364856552000637, P101=0.0009846929853116317, P102=0.0020829457049346446, P103=0.0003122306868934684, P104=7.098097063564018e-05, P105=0.000178747018403926, P106=0.0003046075517892405, P107=0.0004262860813016726, P108=0.0011314853227793703, P109=0.001188543746062017, P110=0.0002992486013999996, P111=0.00018555973915012215, P112=0.0003526859481630968, P113=0.0004772852824954432, P114=0.000260173932929082, P115=0.00011882910301462589, P116=0.00014424858915297706, P117=7.050341036795793e-05, P118=0.00017520438485702734, P119=0.00011827806034721196, P120=0.000163408841013159, P121=0.00018564583580350127, P122=0.00013424173532943865, P123=0.0001638696460745223, P124=8.533225586752767e-05, P125=0.0001754934269103254, P126=0.00011339578209993914, P127=9.157019956367569e-05, P128=8.812459334973862e-05, P129=7.195403784231994e-05, P130=4.332356904735863e-05, P131=7.155809648082858e-05, P132=8.542015684488618e-05, P133=8.401498794944518e-05, P134=8.142238924269258e-05, P135=4.7683366247993424e-05, P136=2.2470697440396705e-05, P137=6.594945071191303e-06, P138=4.479752836190856e-06, P139=3.875908112242498e-06, P140=2.409823088163148e-06, P141=3.043690236454436e-06, P142=1.9541323337434424e-06, P143=5.416125654998421e-06, P144=8.201422780830103e-06, P145=1.2953086602150648e-05, P146=1.1318602954095892e-05, P147=7.967004013973598e-06, P148=9.967293141396766e-06, P149=6.649379775746853e-06, P150=7.730707413803938e-06, P151=4.8875896610014705e-06, P152=5.714211863038296e-06, P153=4.398700175195629e-06, P154=2.987402263477415e-06, P155=8.55609235662262e-07, P156=1.9448151001146905e-06, P157=2.9240291106172026e-06, P158=2.7785192958588253e-06, P159=4.394671572134217e-06, P160=7.468468727976301e-06, P161=8.006991311467469e-06, P162=8.924749510578725e-06, P163=4.972008528626548e-06, P164=3.06589993167565e-06, P165=5.542798069430157e-07, P166=7.307360667713932e-07, P167=1.461103023886083e-06, P168=3.824220674661803e-06, P169=2.343217333257162e-06, P170=1.8701045509575418e-06, P171=1.0793243163180663e-06, P172=3.429248897450231e-07, P173=5.28144472704077e-07, P174=2.1582423811259574e-06, P175=2.3930544579257243e-06, P176=4.676594696243738e-06, P177=3.889106442309038e-06, P178=1.2296661258540753e-06, P179=1.370834757720094e-07),Row( P0=-687.2108711334207, P1=55.23619181179559, P2=0.4376689727167801, P3=-1.5713686686099781, P4=-14.924496484221828, P5=-13.242415345080921, P6=1.3229812980031381, P7=-5.190835159064928, P8=-6.73428321841622, P9=-4.480578322324392, P10=-5.837258583547472, P11=-9.305273533851928, P12=-5.688425406641877, P13=-6.995132579270557, P14=-4.074800438361131, P15=-3.595794985595259, P16=-3.246594688154485, P17=-2.0907971693323466, P18=-4.395269249215697, P19=-0.9163721439658016, P20=-5.084421452172768, P21=-2.626665067652063, P22=-0.8498069911088428, P23=-1.3248857620956473, P24=-4.235415557808567, P25=2.748967324120457, P26=-2.1421775837799, P27=3.2448282604015035, P28=0.5781026840417568, P29=2.2803888789343914, P30=-0.30712616675648785, P31=3.8907812629235963, P32=0.20184342384640475, P33=3.8902180736786325, P34=0.0644474351397576, P35=2.7911583130023865, P36=1.515592511251926, P37=4.387155220372381, P38=0.44018935396284137, P39=0.7006694545159016, P40=0.5921541562369436, P41=0.5748445927248801, P42=0.5863716643576419, P43=0.5852305977199947, P44=0.5826704866217731, P45=0.6235750007594585, P46=0.5633771568827618, P47=0.5320299909604658, P48=0.5447270541767019, P49=0.5503201743000863, P50=0.5697292676830943, P51=0.5637371176486269, P52=5.5091449804386695e-05, P53=3.691725351722837e-05, P54=6.32333671263562e-06, P55=1.0512523156109144e-05, P56=0.00020163759728535215, P57=0.0012629856454805688, P58=0.00276218863798212, P59=0.004561770655122239, P60=0.0070178468188936945, P61=0.007421987249371732, P62=0.0064332175230813315, P63=0.004456128796251424, P64=0.004640403703241478, P65=0.00511691308617042, P66=0.003638353949549292, P67=0.004806901398038044, P68=0.007297344637283401, P69=0.010371457488076267, P70=0.012310172804346589, P71=0.010183708154600974, P72=0.00395678808859629, P73=0.0037193737666791804, P74=0.0026566393047585137, P75=0.002801457596952887, P76=0.005310960877069329, P77=0.006584552000075761, P78=0.01871831096754055, P79=0.027559377797965478, P80=0.013350634629856306, P81=0.010495249366465197, P82=0.012792139652075957, P83=0.015818076296571913, P84=0.0196449295970294, P85=0.015689588545387118, P86=0.006649987890164415, P87=0.006606539273528324, P88=0.00511758175341211, P89=0.0031533630541294585, P90=0.004166018621178007, P91=0.008058267582322094, P92=0.01614612450385124, P93=0.006592930040548385, P94=0.002420960146251177, P95=0.0025115207796609583, P96=0.0038899904832626676, P97=0.005638448182517726, P98=0.0024399901945430682, P99=0.000996371602171034, P100=0.0010924907144807803, P101=0.0013070389146105216, P102=0.0007285513665990204, P103=0.0004708819719931594, P104=0.0006375132900828402, P105=0.00047001424909031336, P106=0.00036949997995057286, P107=0.0007796999696392051, P108=0.0018394890653109874, P109=0.0014801505262031329, P110=0.0005303270748670493, P111=0.0001429407313198141, P112=0.00019765466005702964, P113=0.0002544684675937012, P114=0.00019839332495829958, P115=9.314102263505264e-05, P116=0.00014703580355361308, P117=2.7558907908504416e-05, P118=0.00012924630907636537, P119=0.0001421477306806767, P120=4.451788803505499e-05, P121=0.00014127459139384485, P122=7.296962363949493e-05, P123=8.285431697666669e-05, P124=9.021096027325445e-05, P125=9.75089360521424e-05, P126=0.00012102776196782344, P127=0.00013194178648310025, P128=0.0001898527994150409, P129=0.0003440033065255764, P130=0.000223393032567282, P131=0.00038187064517645894, P132=0.0005251334563489756, P133=0.0004984776391603876, P134=0.0006794016304117221, P135=0.0006012585743757469, P136=0.0003218432855201693, P137=0.00012809563744272553, P138=9.268035347409515e-05, P139=6.190805168817603e-05, P140=5.7252965899573865e-05, P141=0.00010321610148318403, P142=0.00011213706976986793, P143=0.00017654417979476764, P144=0.0001731432990587107, P145=8.940252050332398e-05, P146=5.724234393891395e-05, P147=3.218472147636648e-05, P148=3.2097961865559486e-05, P149=2.3804845186836985e-05, P150=2.2294809555423563e-05, P151=3.859332405724635e-05, P152=7.334220700133993e-05, P153=0.00018903150812717205, P154=0.00025378656540862783, P155=0.00035617585371078536, P156=0.0002659194325171956, P157=0.00032622146109022326, P158=0.0002684470437393492, P159=0.0004899362708834571, P160=0.0005493164644668855, P161=0.0002483197226436918, P162=0.00014105811450486886, P163=7.572480518250227e-05, P164=5.096593678763239e-05, P165=0.0001177316238176911, P166=0.0002174923464568046, P167=0.00022152389526697088, P168=0.0002746313616218707, P169=0.00017944205613087095, P170=0.00010972397964388673, P171=3.706360720004903e-05, P172=2.171207464488039e-05, P173=4.581770739078223e-05, P174=8.940839878620125e-05, P175=0.0001066883205076116, P176=8.235032609625813e-05, P177=6.330609595942e-05, P178=2.7566402413788496e-05, P179=2.6845980058406852e-06),Row( P0=-672.0820101253116, P1=67.15013190568128, P2=-8.77460024415069, P3=5.841891810020596, P4=-16.954733726438032, P5=-14.818215095485153, P6=-6.178252737208738, P7=0.19608336118782418, P8=-0.00854219447409402, P9=-13.294989770846835, P10=-3.834757559273047, P11=-5.433768265275626, P12=-6.096883565863585, P13=-1.3393556415355459, P14=-0.8815175042248818, P15=-3.6406951622480936, P16=2.1382923110138865, P17=0.9734078378218046, P18=-7.4791322612831275, P19=-2.030290891726511, P20=-7.870397403876902, P21=-1.8128933702105927, P22=-1.4325693118922715, P23=-2.645486476826264, P24=-2.260670401454807, P25=4.2484788609007245, P26=1.0914341721714502, P27=6.044664867020874, P28=1.3898839497149493, P29=5.178098173813488, P30=-1.761042242633926, P31=7.260325431606505, P32=-0.49239184235590516, P33=6.986417380763851, P34=-0.8367536022473323, P35=5.6741769334036585, P36=4.434410390221416, P37=6.33987111093887, P38=-1.1797684564356037, P39=-0.057263348042241174, P40=0.7063883644092027, P41=0.6330072801178344, P42=0.646089235760443, P43=0.6617466097762622, P44=0.5880093464965638, P45=0.6252037112182521, P46=0.6326534297599724, P47=0.5419138526724792, P48=0.4810189384318832, P49=0.5769884906907555, P50=0.6809341930173569, P51=0.717676840654409, P52=3.220832789921739e-05, P53=2.9712661638135846e-05, P54=1.1946741906977455e-05, P55=4.745936423018056e-06, P56=1.0207225013030177e-05, P57=0.00031448945186350185, P58=0.0032707240704076355, P59=0.005936275562495608, P60=0.004544751127254631, P61=0.00779562270561269, P62=0.00631422262256281, P63=0.0010345209774034643, P64=0.000180218144622225, P65=0.0015322185129080298, P66=0.004309727091657639, P67=0.004626497530560934, P68=0.0024494317539410235, P69=0.002927969396319045, P70=0.0062861300450393134, P71=0.003749983520228489, P72=0.0029484701197614147, P73=0.0029124846476487735, P74=0.002150684979254955, P75=0.0014843565185849246, P76=0.0014062954278398379, P77=0.001130220280341045, P78=0.002475360718259456, P79=0.005843062751274401, P80=0.004908693303785347, P81=0.006098392019344972, P82=0.010925399997633218, P83=0.009049364507296412, P84=0.006139556964364422, P85=0.0035476785234644804, P86=0.0037790139642950317, P87=0.003319608343268187, P88=0.007555573234246311, P89=0.004952235341883, P90=0.0010236478565152688, P91=0.0010555730291455778, P92=0.0014870009774632272, P93=0.0007048620500550795, P94=0.0065797113414160216, P95=0.005553240181623521, P96=0.0014211181066619573, P97=0.0006115933928196829, P98=0.0007816768275977498, P99=0.0030862438907606703, P100=0.0032456311963929307, P101=0.0030144486362242914, P102=0.0010430723686883903, P103=0.0002951900310277613, P104=0.0001378948833745931, P105=7.496155607856978e-05, P106=0.00012626887164525428, P107=0.00021014851657187396, P108=0.000633212676401029, P109=0.0012128898341207523, P110=0.0005343714712619846, P111=5.1958248214712814e-05, P112=0.0002879797854009035, P113=0.0005689425684287729, P114=0.0001039078382099526, P115=9.92338830313254e-05, P116=0.00020258575869903326, P117=3.2285294292001284e-05, P118=0.00010741926427300781, P119=0.00010967706039180739, P120=6.12255915480619e-05, P121=9.076155976323898e-05, P122=8.372554227638142e-05, P123=0.00010862828207028217, P124=4.5244923090782586e-05, P125=5.80289636988691e-05, P126=4.055684350985863e-05, P127=3.531345977399516e-05, P128=3.222716233467153e-05, P129=1.762194662827791e-05, P130=1.699695170418299e-05, P131=1.5058253541496346e-05, P132=2.083346829507421e-05, P133=2.4541679599702307e-05, P134=2.5324607186739852e-05, P135=2.4497031152852175e-05, P136=1.3964800622206075e-05, P137=3.5295825031627584e-06, P138=2.520048765579248e-06, P139=2.622741037214839e-06, P140=1.6682280536456299e-06, P141=3.100384594961036e-06, P142=2.8763281906337444e-06, P143=5.0305964930643665e-06, P144=1.128672168583927e-05, P145=1.4999317279992738e-05, P146=1.0814986059610796e-05, P147=7.174241608250285e-06, P148=7.613077288508258e-06, P149=6.460340300225044e-06, P150=7.390588442086697e-06, P151=6.935935490137757e-06, P152=5.057985129819347e-06, P153=4.572220186869448e-06, P154=3.1518898730395224e-06, P155=3.5232578265724413e-06, P156=7.08414492885483e-06, P157=9.654173706659308e-06, P158=8.274819642886073e-06, P159=8.843673432574013e-06, P160=7.2538991038759404e-06, P161=5.130163329282904e-06, P162=3.3262647411065196e-06, P163=1.5048186080579937e-06, P164=8.969264646844109e-07, P165=8.116043129584934e-07, P166=3.6073362910612513e-06, P167=3.546002762818336e-06, P168=3.8112364109096073e-06, P169=3.0237007986432966e-06, P170=1.1273360193991254e-06, P171=5.448744342818929e-07, P172=1.197693836174378e-06, P173=1.6538869954491285e-06, P174=2.682241281231925e-06, P175=5.5492802400713675e-06, P176=4.248639241715272e-06, P177=3.245227586033928e-06, P178=1.6142313643552134e-06, P179=1.8179632359775e-07),Row( P0=-675.4499875977824, P1=58.563531698951245, P2=9.09165011521554, P3=4.373538729090337, P4=0.014258064885365788, P5=-3.1590464885315623, P6=-1.6709803183264649, P7=5.727659735916515, P8=3.194596648050043, P9=3.1547609519726385, P10=-2.538006498488481, P11=-0.9523645847260992, P12=0.5462554205298692, P13=-2.5442575098716222, P14=-0.10678451173708625, P15=0.849155114123806, P16=2.2719851089353686, P17=-1.2758286245989698, P18=-1.1592544811804402, P19=-0.4346029295628566, P20=-4.004551525172887, P21=-1.401989109190121, P22=-0.699543574590118, P23=0.4312369016878189, P24=-0.770293943038627, P25=4.646747041095916, P26=1.0463437571296867, P27=3.582783765189563, P28=2.6210566552048977, P29=1.7301833674253795, P30=1.4748871208763628, P31=5.476566732283534, P32=2.273764828593473, P33=4.980655331131334, P34=1.6187385432493215, P35=2.8240349799078537, P36=0.2608460402245816, P37=2.0667442693326126, P38=1.1946725781217522, P39=1.7166285760547577, P40=0.7520543930679155, P41=0.7279785284726027, P42=0.7076104941651329, P43=0.6959220448566223, P44=0.7392169643504894, P45=0.6943116310118694, P46=0.6152163521237674, P47=0.6182379474816574, P48=0.6470107437375494, P49=0.695815837536903, P50=0.7150043575539946, P51=0.7394029459653071, P52=5.707018458501594e-05, P53=1.490641290603037e-05, P54=2.3592831800058675e-05, P55=1.4070828356080714e-05, P56=8.159497177603272e-05, P57=0.0006798705522822164, P58=0.0026701747477061396, P59=0.011508114870139488, P60=0.028902698899935974, P61=0.06027864916733723, P62=0.04909950686570323, P63=0.009035630354905249, P64=0.0010118053438704258, P65=0.004088421705963117, P66=0.009357980530269818, P67=0.01955050899915639, P68=0.04022446808561083, P69=0.045450054634141736, P70=0.023692042963930703, P71=0.01542866351517435, P72=0.020905412178892066, P73=0.016038576466995056, P74=0.005790505827744447, P75=0.004022199248546137, P76=0.004442585165458537, P77=0.015450186694977715, P78=0.0399017447352781, P79=0.05609536445570019, P80=0.027623854268371436, P81=0.028724285040206802, P82=0.04670142933372645, P83=0.045573026640503125, P84=0.030120080259638225, P85=0.017733329569640164, P86=0.0056777726904411125, P87=0.0030850269040260084, P88=0.003045905178664294, P89=0.00423501588109571, P90=0.010899128823506948, P91=0.010342227869758075, P92=0.024805431773750792, P93=0.015451277632214771, P94=0.001008842162688463, P95=0.001261513396787021, P96=0.002195904204513377, P97=0.004893599869118071, P98=0.006251567841037007, P99=0.006588196521791059, P100=0.0021543200923252784, P101=0.0006392948661001633, P102=0.0006725644182352896, P103=0.0009303154300298798, P104=0.0007257638816701919, P105=0.0005613639852742987, P106=0.0003633806731666852, P107=0.00037094200361475175, P108=0.0012586799875575648, P109=0.00335878731017508, P110=0.0031199093601373433, P111=0.0017142762964477154, P112=0.0010232338324747895, P113=0.0008475806708338004, P114=0.0012327397736084772, P115=0.0006702543683760678, P116=0.0005227882998677877, P117=0.0006262927679805613, P118=0.0003722630533053574, P119=0.0004170959765355508, P120=0.00032001479689053807, P121=0.00018841161019362535, P122=0.00016436343063350563, P123=0.00019190982478956928, P124=0.0003624189406362745, P125=0.00029482466752546525, P126=6.132729167965241e-05, P127=0.0001555256169180039, P128=0.00026024871548949516, P129=2.9370406269919614e-05, P130=2.773595597381074e-05, P131=6.700416926761376e-05, P132=9.274022237219331e-05, P133=0.000178170585332009, P134=0.0001354332994058527, P135=9.548385563352309e-05, P136=6.965005509035585e-05, P137=5.564831682854736e-05, P138=1.4413817002467067e-05, P139=1.3469101850364493e-05, P140=1.2737772649131406e-05, P141=1.7556295085291613e-05, P142=3.18742543401289e-05, P143=4.7850750793781635e-05, P144=5.01515866614194e-05, P145=4.9322734415208565e-05, P146=7.805581047517494e-05, P147=5.163931367863598e-05, P148=3.0041539017392785e-05, P149=4.766952071059699e-05, P150=0.00014287483920524192, P151=0.00020318805526416033, P152=0.00045708785591155074, P153=0.00044394952529565367, P154=0.0002770405150535111, P155=0.00027879955071620965, P156=0.00038388763494319445, P157=0.0004861609616729574, P158=0.00037896053848380476, P159=0.000312288089575962, P160=0.0004305729286864175, P161=0.0003419852712607082, P162=0.00034140041800730624, P163=0.0002567347485738615, P164=0.00010624478556186383, P165=0.0001737047858140828, P166=0.00011587961213327074, P167=0.00017641915823607285, P168=0.00014053951948078545, P169=0.00011730567097925165, P170=7.549308774703e-05, P171=7.98919244404066e-05, P172=0.00010012948363768645, P173=5.285388520451723e-05, P174=5.461246828834275e-05, P175=9.438985819778272e-05, P176=8.518342273503666e-05, P177=5.7185743397231986e-05, P178=2.9565767524065543e-05, P179=3.1902894563388276e-06),Row( P0=-736.9680956930696, P1=75.23965985361447, P2=19.080003158143914, P3=16.051614673222325, P4=5.5574146105587845, P5=4.306639501658779, P6=4.657572490703597, P7=6.079015811320922, P8=4.1000305504285475, P9=1.3560089657198844, P10=-3.982772527253173, P11=-1.1259591481514242, P12=3.6297381789863405, P13=-3.746182534131199, P14=-2.6837622927801634, P15=1.5223608582656432, P16=1.8667429786204064, P17=0.12585932253160717, P18=-1.1511523636224354, P19=0.7089798925747545, P20=-3.1639107927095993, P21=-2.1503127610879584, P22=-2.797935123302773, P23=-1.8728981542197547, P24=-3.652657463181341, P25=0.038517818563991724, P26=-4.269255906794705, P27=-1.9193209384361907, P28=1.804083639820514, P29=2.4849297146495566, P30=1.6209883126273168, P31=7.306334998263252, P32=3.018831342049196, P33=5.652710044112254, P34=1.2314776223510124, P35=4.693422884984833, P36=2.0575048814832084, P37=3.4328127590081228, P38=1.4477920728634677, P39=4.016981849727912, P40=0.6961133649944385, P41=0.6829564235283047, P42=0.6417344537220695, P43=0.5850889222068345, P44=0.6005521154030905, P45=0.6585834291379015, P46=0.691292294814469, P47=0.7039566760976236, P48=0.6766192113488522, P49=0.669627540538275, P50=0.6705048948762377, P51=0.6990897425257249, P52=9.741991032092129e-05, P53=3.4248683272454676e-05, P54=2.0389456635681715e-05, P55=1.2612598107578993e-05, P56=0.0005275749472062034, P57=0.0031250883811475747, P58=0.004808729599712892, P59=0.009515816013202607, P60=0.007491849445163315, P61=0.0009362804224260356, P62=0.0008447314485483459, P63=0.0011907779661605613, P64=0.0007517807945174878, P65=0.003650794437448596, P66=0.010876723361339487, P67=0.01797522602279442, P68=0.01053760098906602, P69=0.004248365760668749, P70=0.0005736172694101417, P71=0.00018782402696077875, P72=0.0003752974620856712, P73=0.0003174272523446566, P74=0.000703492457023622, P75=0.0012877722470293878, P76=0.0009738024612550303, P77=0.0017399671611864288, P78=0.0009403200220515591, P79=0.00045925862333202975, P80=0.0003330502398943101, P81=0.0002013179117905585, P82=0.0003172030670530569, P83=0.000563272279916969, P84=0.0008088667497432111, P85=0.001486234737024157, P86=0.0008320642658090112, P87=0.00017222406197115868, P88=7.904251401260197e-05, P89=3.918091720771473e-05, P90=4.952234458025532e-05, P91=0.00018948322599865236, P92=0.0010056211459743985, P93=0.0007034668012494006, P94=0.00010737169621879059, P95=4.428853235841554e-05, P96=9.856705188454152e-05, P97=0.00032760774363390315, P98=0.00107936969598678, P99=0.0002980257166255251, P100=3.089795219911106e-05, P101=2.8301238789891817e-05, P102=9.30746138166725e-05, P103=0.0002016210958682445, P104=4.215427026118431e-05, P105=1.0642713259654648e-05, P106=4.4713546709756144e-05, P107=0.00011573665701396703, P108=4.6328089122032536e-05, P109=1.163781204664338e-05, P110=5.528015586017611e-05, P111=4.6779713820858515e-05, P112=2.108433005108108e-05, P113=6.387322368962813e-05, P114=3.945265088825754e-05, P115=4.639269243648492e-05, P116=3.6630309719936824e-05, P117=2.990130908383481e-05, P118=2.228251847940305e-05, P119=4.458772214368433e-05, P120=2.1264951973100096e-05, P121=1.5399997444563972e-05, P122=1.4507757312591445e-05, P123=8.265060283788937e-06, P124=1.2538991685178524e-05, P125=8.53252445904391e-06, P126=2.898384548020373e-06, P127=4.104288552742788e-06, P128=4.226449294164718e-06, P129=2.5902532080881423e-06, P130=4.531609364418737e-06, P131=6.534441729946644e-06, P132=7.171353327327525e-06, P133=1.1233582827838272e-05, P134=1.476660134493777e-05, P135=1.2869057818583232e-05, P136=9.811146340702975e-06, P137=5.073036663337777e-06, P138=5.324624534576761e-06, P139=7.173389611759741e-06, P140=1.1202266373364286e-05, P141=1.7250861217513624e-05, P142=1.51852269202519e-05, P143=2.03762240047204e-05, P144=2.3721969652036503e-05, P145=2.6642067446404564e-05, P146=1.1666813799611612e-05, P147=5.504154581757608e-06, P148=6.810833532493025e-06, P149=1.7708414863001077e-05, P150=3.8875462757226984e-05, P151=8.683844429098756e-05, P152=0.00012047753126891096, P153=7.866422568940661e-05, P154=6.422621674721141e-05, P155=3.7964402754476326e-05, P156=3.771998021617671e-05, P157=4.280457948570994e-05, P158=6.367255125935042e-05, P159=8.620536782726578e-05, P160=8.847726123839955e-05, P161=7.900263822663761e-05, P162=6.323337735491078e-05, P163=4.7160336623925175e-05, P164=3.961007147817533e-05, P165=5.8887765613448176e-05, P166=4.340846131653212e-05, P167=4.303896186699607e-05, P168=2.820511426839313e-05, P169=1.7922035520796034e-05, P170=8.79269479695988e-06, P171=1.1918551218601784e-05, P172=1.3522768952793133e-05, P173=1.3445262848353256e-05, P174=2.3902705453721104e-05, P175=2.9458549701011437e-05, P176=4.013434525349591e-05, P177=3.1726478790059555e-05, P178=1.222726333904826e-05, P179=9.628930352897386e-07),Row( P0=-675.9139560425491, P1=66.63005553550363, P2=7.993927340113583, P3=5.7577142148300755, P4=1.7806886652388998, P5=-2.0454308687332303, P6=-1.06024702189129, P7=3.6482470924812165, P8=1.6888873639868291, P9=1.8980819201203807, P10=-2.561961568213468, P11=-0.643925960988536, P12=1.2885106120302843, P13=-3.0520842551165317, P14=-1.2742145942814083, P15=1.0802776016701277, P16=0.6457743146087914, P17=-1.6257436038191444, P18=-0.9116493634873714, P19=0.135047992654192, P20=-0.48600732451340756, P21=1.1766908637333118, P22=1.2700484830921737, P23=0.9916459373665081, P24=-0.0010152109762507412, P25=3.09871435782675, P26=-1.1110313615579872, P27=1.8949287438470057, P28=1.6962854996811751, P29=5.123136125276371, P30=3.144240444197982, P31=5.248452243565453, P32=0.016233846702180473, P33=1.7681309170445771, P34=-1.0338132241165972, P35=1.9203651101965984, P36=0.6865340915677678, P37=1.2041467063525466, P38=0.06202446086211345, P39=0.2149144846726355, P40=0.7238227559612892, P41=0.7209201442162892, P42=0.7302402864811941, P43=0.6964597223613821, P44=0.7329855667378651, P45=0.7275161229325137, P46=0.7757976427154267, P47=0.7337964868897222, P48=0.6844269091589974, P49=0.6695015815838915, P50=0.6727364300267012, P51=0.719872514217351, P52=0.00039694834310559504, P53=0.00011480201342071985, P54=2.10294324853938e-05, P55=1.1617370054905592e-05, P56=0.00046471094081700374, P57=0.0043356315433596025, P58=0.004012984825095135, P59=0.009231566830074424, P60=0.03098935455370289, P61=0.03646358364412017, P62=0.06894885273589485, P63=0.08457764425432393, P64=0.016263023830719498, P65=0.004284366829319487, P66=0.011632025147099214, P67=0.02097726309358173, P68=0.023304273796707617, P69=0.0442485129496966, P70=0.03678554093493905, P71=0.00953686117303984, P72=0.0032310124681942582, P73=0.005556578567438645, P74=0.035989178537872715, P75=0.06337133187428927, P76=0.02865961164741556, P77=0.013491214425239477, P78=0.013867209006707964, P79=0.011011952679927866, P80=0.005902958368184766, P81=0.004082253355174025, P82=0.005404072647509691, P83=0.004564099677493958, P84=0.006519479980532875, P85=0.010890425532406251, P86=0.013469894336469983, P87=0.00870438852333264, P88=0.006644106466544995, P89=0.002713520753573055, P90=0.005225912238009528, P91=0.021599018353707068, P92=0.00874642668409765, P93=0.002963459754478212, P94=0.0102996313725479, P95=0.015224643218938693, P96=0.007164713993364141, P97=0.0057728486670391614, P98=0.002933561272386906, P99=0.0009009979741936094, P100=0.0065537576313696565, P101=0.008253408008732161, P102=0.003954221415641204, P103=0.0012598061670007778, P104=0.00028418466305019964, P105=0.0005714683224711417, P106=0.0007823151877861186, P107=0.0004552428751793982, P108=0.00048770374943635914, P109=0.0012179066686732107, P110=0.001387879552040411, P111=0.001141162588628931, P112=0.0009840732001070787, P113=0.003262053456006757, P114=0.0009366134234854186, P115=0.0010088694059852937, P116=0.0014714946528364688, P117=0.0003065681831711341, P118=0.0007463885921121272, P119=0.0004068606065025339, P120=0.0003174382786983166, P121=0.0005036734451798466, P122=0.00015241397585372262, P123=0.00013955714663261727, P124=0.00011639303649903961, P125=0.00016357095572662757, P126=8.324800308518002e-05, P127=6.187826209650676e-05, P128=9.434933707949153e-05, P129=3.989172078914269e-05, P130=4.0194319874847945e-05, P131=5.8867359441447056e-05, P132=6.0308651932424195e-05, P133=9.202623667276545e-05, P134=0.00013070661405113147, P135=8.820444453310549e-05, P136=9.247947778838169e-05, P137=6.990909681728314e-05, P138=5.546416562962563e-05, P139=6.193513274329751e-05, P140=5.50051276824665e-05, P141=3.8508734297599225e-05, P142=2.8659386048856813e-05, P143=6.440062508313012e-05, P144=5.2323401727879485e-05, P145=4.9643171674852874e-05, P146=3.996575698831621e-05, P147=2.1338906368698137e-05, P148=1.1960249052918914e-05, P149=1.870309795392307e-05, P150=4.1965573110902156e-05, P151=5.588662411687558e-05, P152=5.477754565220608e-05, P153=5.856854606765944e-05, P154=0.00010357003424063874, P155=5.896290302504158e-05, P156=0.00014668565540101563, P157=0.00010352322170970697, P158=5.8026822026867036e-05, P159=9.457200238001803e-05, P160=0.00010892479600250686, P161=9.66916259061104e-05, P162=3.268039562293744e-05, P163=3.063107353760928e-05, P164=2.621864207678565e-05, P165=2.734072779219066e-05, P166=3.657664264374646e-05, P167=2.64055067012384e-05, P168=2.037075350659527e-05, P169=1.2461006554733674e-05, P170=1.0155535918885567e-05, P171=8.520025478251529e-06, P172=8.780837820368503e-06, P173=1.3006938090176156e-05, P174=1.8555943001651894e-05, P175=2.2812824034896028e-05, P176=3.482514517416009e-05, P177=3.097661199465943e-05, P178=1.4906804215859591e-05, P179=9.247378996298818e-07),Row( P0=-613.8749115119305, P1=66.21471088674409, P2=5.302596371111711, P3=-3.966607468243214, P4=-8.975219696800425, P5=-0.9093163329743825, P6=-6.118892962545666, P7=-6.262919742620165, P8=-4.5177696605227675, P9=-3.6328454885752466, P10=-8.29234723189516, P11=-7.742510793217825, P12=-0.7978397922568452, P13=-6.967472734529516, P14=-2.336078464327715, P15=-1.8097095611974645, P16=-2.2892898394551726, P17=-3.052572362480079, P18=-1.211179905301803, P19=-0.6522669969811987, P20=-2.7065129301297133, P21=-2.410256524324323, P22=-0.6672722509109623, P23=2.050064018755622, P24=0.7684324929038753, P25=4.390692679794704, P26=0.4444733624170123, P27=3.3407740069428384, P28=3.213314365345644, P29=4.551664531144591, P30=3.744903793227128, P31=6.97358739620824, P32=1.2057280415372054, P33=1.8923038877510432, P34=-0.5339572931688816, P35=1.654820681736838, P36=2.1139312317325976, P37=2.531920743332688, P38=0.34416424999347933, P39=-0.6191274008905905, P40=0.699725439491069, P41=0.7073798018303569, P42=0.7253437950094416, P43=0.7050478373093981, P44=0.6917887424819292, P45=0.7039816518815762, P46=0.7009610348967423, P47=0.6345888618838147, P48=0.6033375793428071, P49=0.6543863403360269, P50=0.6762123773008153, P51=0.6683107601122459, P52=4.882879542218571e-05, P53=2.5900656242743146e-05, P54=1.995633787585478e-05, P55=4.2551990410531934e-05, P56=0.00018928185796112816, P57=0.0008996389117610643, P58=0.0029419318874888186, P59=0.011975195969730958, P60=0.04900036593352918, P61=0.08920303798918683, P62=0.10547870889259316, P63=0.056923256731225134, P64=0.005252729719271104, P65=0.004030356410843197, P66=0.0091409063364824, P67=0.01366549176471433, P68=0.01813189670952628, P69=0.04191966461323086, P70=0.04480339764407765, P71=0.02737277321662998, P72=0.03070955786299974, P73=0.02967014656506603, P74=0.017401162240579443, P75=0.012505902317019313, P76=0.003919321049440289, P77=0.006935495391345369, P78=0.020507603944060742, P79=0.03498796428491316, P80=0.020218482896740744, P81=0.013878841068882899, P82=0.02299598241449045, P83=0.03126334091516489, P84=0.03490561637202522, P85=0.03467110517558805, P86=0.017729301894760994, P87=0.01305566275340965, P88=0.0120320648232617, P89=0.008797394707902185, P90=0.009246142466728488, P91=0.01569321209974473, P92=0.05168268761184135, P93=0.06115080540015123, P94=0.13053057770997545, P95=0.1757790048001926, P96=0.033374031475640324, P97=0.006169074485631892, P98=0.009193925088773777, P99=0.018221974013435335, P100=0.04742313467148593, P101=0.10931508697948428, P102=0.1024267339972394, P103=0.016171251339157297, P104=0.007220615323956436, P105=0.010911402160384352, P106=0.016943668410767304, P107=0.01586575246411889, P108=0.0084198633732713, P109=0.004599530460698621, P110=0.0027106293934329568, P111=0.0012314432209095333, P112=0.0009035068809152921, P113=0.0015352826709594408, P114=0.0006915439966129205, P115=0.0016327517988967875, P116=0.000900217129882397, P117=0.0005467580034299474, P118=0.001620018775160141, P119=0.00036531158501517974, P120=0.0005461801489885612, P121=0.0007252703780163011, P122=5.1439569107353135e-05, P123=0.00017604612935138173, P124=9.76369798870251e-05, P125=9.727049293711033e-05, P126=6.175773826601753e-05, P127=8.93178905716164e-05, P128=7.194494851018743e-05, P129=2.828276679402275e-05, P130=4.090366883853102e-05, P131=6.720532675927738e-05, P132=5.471822490313541e-05, P133=0.00021755785061942112, P134=0.00015873842618353856, P135=0.0002597773006231777, P136=0.00023376217512438577, P137=0.00018228505986947783, P138=9.712720681384215e-05, P139=0.00012614901138822957, P140=0.00011553672761762189, P141=8.916381933746895e-05, P142=8.552324110316762e-05, P143=0.00013278521807737849, P144=0.00011364619754099734, P145=8.71602183197949e-05, P146=6.19141365299656e-05, P147=6.323564606007546e-05, P148=3.708214186848961e-05, P149=4.648322386686334e-05, P150=7.660109035898388e-05, P151=0.00014723291359912557, P152=0.0004576310420464339, P153=0.0004211995366627231, P154=0.00042311583504633146, P155=0.0005104530775467617, P156=0.00025686133175176104, P157=0.0002494862148339943, P158=0.00030983833540158335, P159=0.00041571713948480626, P160=0.0004345924130240914, P161=0.00043292877959564017, P162=0.00036363797466817504, P163=0.00022809070149484663, P164=0.00015879437195772167, P165=0.0001500433410421061, P166=0.0001169238878821763, P167=0.00013514534658158446, P168=0.00011009687774404319, P169=5.2506777038431864e-05, P170=2.7314421736604064e-05, P171=3.474021166712519e-05, P172=2.7122336946062837e-05, P173=3.793196818425246e-05, P174=2.9474406453692104e-05, P175=4.828776472446911e-05, P176=0.00011345576172202219, P177=0.0001154554479586155, P178=9.388292999678167e-05, P179=1.6292637312202407e-05),Row( P0=-637.0522783388278, P1=61.81517316254393, P2=8.45156017757041, P3=3.2668871226605956, P4=-1.1132562240637531, P5=0.8009177746624322, P6=-1.887848252463136, P7=-1.9817640937698462, P8=0.9751984604252696, P9=2.8441066997495676, P10=-3.713170144779277, P11=-3.5820282377773, P12=0.43315387225327423, P13=-2.3927979568503273, P14=-2.4281139897299955, P15=-0.6672493685533094, P16=-0.29360298997401896, P17=-1.1848630981378114, P18=-3.0987342248115857, P19=0.30737609565653545, P20=-3.3410851984414207, P21=0.5002047810128951, P22=1.3636431165839737, P23=3.518656107117104, P24=3.0913095550635186, P25=8.38806656275291, P26=3.2447410592175454, P27=4.900077610924761, P28=3.1987507625751386, P29=6.265900546171791, P30=4.0049888906021875, P31=6.637132684997531, P32=0.7202952858281291, P33=4.244981132357151, P34=-0.25036313446570363, P35=0.9782541678825448, P36=1.4285019568542698, P37=1.6184455454644977, P38=-0.538843949076545, P39=-0.421976785065019, P40=0.693121713875395, P41=0.7169107810025312, P42=0.7079480916516492, P43=0.7222291828325617, P44=0.7445976878545482, P45=0.7610369232387969, P46=0.7153078652102265, P47=0.6213374726557251, P48=0.5986111069952033, P49=0.6360676319810973, P50=0.661498786155081, P51=0.6500878044087138, P52=0.019529780291850082, P53=0.0006337679314696328, P54=7.8856616515453e-05, P55=3.7279275430649886e-05, P56=0.00016918123643862006, P57=0.0009645098890065233, P58=0.0026048569285444757, P59=0.017979936801694053, P60=0.08257331094073714, P61=0.143895250945046, P62=0.18192296533489338, P63=0.10074536632849043, P64=0.006222445299959998, P65=0.0021297657309714076, P66=0.0042342852630031, P67=0.007575761980188232, P68=0.011377488694127184, P69=0.023523393216085337, P70=0.035127951482698906, P71=0.02525038172211264, P72=0.012698276265032962, P73=0.024578585225902946, P74=0.05752643034800216, P75=0.020229336822507722, P76=0.004277628169260474, P77=0.006160960762926857, P78=0.011507019005532563, P79=0.017533906367169373, P80=0.034243773666139586, P81=0.03425311031511291, P82=0.01360282865125941, P83=0.005771029971718515, P84=0.010723538257640207, P85=0.023003883446319394, P86=0.011302875237859401, P87=0.005876415855934314, P88=0.006983946606084438, P89=0.0048637540366623586, P90=0.006995732295592548, P91=0.011531520399592576, P92=0.008852782545554761, P93=0.008182482021293768, P94=0.0464722838072674, P95=0.024039323420235608, P96=0.003453575914678402, P97=0.003851508747369695, P98=0.002985509616987102, P99=0.0014262420338443944, P100=0.007302702050906359, P101=0.02341729241589527, P102=0.0069493258626894166, P103=0.0015343374320368152, P104=0.0009253206049556215, P105=0.0021233044126639337, P106=0.0052307966035603694, P107=0.009947092573117499, P108=0.0022818027947435855, P109=0.0012391718334459673, P110=0.0017181131247843897, P111=0.0036008288451097195, P112=0.0022350120717487135, P113=0.0011723562709586516, P114=0.0007610509022356001, P115=0.000913965907142768, P116=0.0011424337687130322, P117=0.00048825803376516914, P118=0.00038791673744328404, P119=0.0008290225055052151, P120=0.0003724486005859883, P121=0.00011277406688978202, P122=0.00014417153159496142, P123=5.6686404364950573e-05, P124=7.574406338294027e-05, P125=6.88143253804186e-05, P126=3.7072186166570075e-05, P127=0.00013009352502683756, P128=0.00013234821932378427, P129=4.338676303810286e-05, P130=0.0001234338731106417, P131=9.819803041417476e-05, P132=0.0001335075916380394, P133=0.00019696531463683458, P134=0.00018275525855181222, P135=0.0003034900146802526, P136=0.00028332909156877234, P137=0.0001966571084770354, P138=0.00018507205275365772, P139=0.00014986436326863196, P140=0.00010280395641108287, P141=0.00013405869686893002, P142=0.00023643891613579104, P143=0.00039624157931861747, P144=0.00017303009011240846, P145=0.0002087789731081984, P146=0.00013459095073948605, P147=7.97576592242576e-05, P148=8.68603550182388e-05, P149=8.324898893521993e-05, P150=5.098693274568053e-05, P151=5.695623107462733e-05, P152=6.454321122445101e-05, P153=8.295130602288122e-05, P154=7.173897984803687e-05, P155=7.60818132197466e-05, P156=0.00010773071266188979, P157=0.00015891615172290526, P158=0.00026350723585438925, P159=0.0002008636702470232, P160=0.00020886211336646095, P161=0.00018195030246034334, P162=9.180947096860678e-05, P163=0.00010205331022065618, P164=0.00010123911065544781, P165=8.349060522645283e-05, P166=8.582380230531658e-05, P167=0.00010201512619555291, P168=0.00012300772901625184, P169=8.085688024445041e-05, P170=6.0302964669294816e-05, P171=5.231546996530365e-05, P172=5.8553028384571643e-05, P173=5.705487604661216e-05, P174=9.851931004484716e-05, P175=0.00010372015988576658, P176=0.00011281820112938058, P177=9.538585823189169e-05, P178=4.27820682382476e-05, P179=3.358679872813462e-06),Row( P0=-623.5189512945558, P1=57.42842762586939, P2=2.5269923845095246, P3=2.0238215433507816, P4=-3.6844337055706253, P5=-4.315700346791374, P6=-4.750966060725925, P7=-0.5711576808805786, P8=3.5706060572476654, P9=-0.2594124421175009, P10=-2.6245189527712287, P11=-2.0430524853289507, P12=-0.812132556012865, P13=-3.477924891825543, P14=-2.396785289824167, P15=1.2383579159837046, P16=-0.914307910854396, P17=-2.537143960096219, P18=-3.583426779015411, P19=1.2454963308188032, P20=-3.4597020090873545, P21=-0.7684957785088187, P22=0.054403365522338605, P23=3.2672440756909107, P24=2.1059701481176005, P25=7.341421272847227, P26=3.201665540920795, P27=4.038884553063104, P28=4.471768554446741, P29=4.8283863066598185, P30=2.259209505575241, P31=3.797054609064096, P32=-0.8678829043413483, P33=1.0760051660766834, P34=-0.15152679625076854, P35=2.825771448871139, P36=0.8045815555365718, P37=-0.3805227095549753, P38=-2.7157992912278566, P39=-1.327401090886656, P40=0.6839041605540067, P41=0.6661736080848374, P42=0.722295267573795, P43=0.7059087145292259, P44=0.721157059156646, P45=0.7065485447128595, P46=0.6890228668493017, P47=0.6723101524639551, P48=0.5800428873141904, P49=0.6001438296961004, P50=0.6577159585744984, P51=0.7020484282987477, P52=4.8666792229969545e-05, P53=7.046015687323154e-05, P54=0.00011380363512715126, P55=4.071971418236621e-05, P56=0.00011419973072078562, P57=0.0006637449357041598, P58=0.0025644113954280765, P59=0.009188369335840696, P60=0.05894942264382834, P61=0.17240462602970394, P62=0.24177634587973962, P63=0.18274997063566592, P64=0.03815670116831349, P65=0.004705807763481168, P66=0.004068367092586378, P67=0.0048486511371207425, P68=0.009951537108947608, P69=0.027899869878195257, P70=0.04979889167954196, P71=0.04384305050120169, P72=0.04460258029662034, P73=0.03569211728134336, P74=0.031779384450933854, P75=0.042734496846075755, P76=0.019852313054882394, P77=0.008102878081010447, P78=0.015815173878547587, P79=0.03969982657753311, P80=0.060438422283403424, P81=0.04835554328479167, P82=0.06955829226549355, P83=0.044758340136393224, P84=0.018660323804572675, P85=0.020447998343543217, P86=0.023262215232329242, P87=0.03848384987855261, P88=0.02296919028833339, P89=0.006681951737668535, P90=0.008245142742054578, P91=0.0282392734593898, P92=0.036566232446855494, P93=0.025312942903140966, P94=0.06267823752466489, P95=0.12740656038934162, P96=0.1517807273719973, P97=0.021513265287873846, P98=0.0023736079268305665, P99=0.001144687205601118, P100=0.003116531488042241, P101=0.00985125737866819, P102=0.02858738978243875, P103=0.03965406421754465, P104=0.0048132544439790275, P105=0.0023646743922637703, P106=0.007253813933221708, P107=0.011562157695273767, P108=0.03254419973651414, P109=0.01605211686107503, P110=0.0033573562011104687, P111=0.0014507722846231128, P112=0.0012186805136540394, P113=0.0018036158034894553, P114=0.0006165777994890295, P115=0.0007851994090937551, P116=0.0018499212376992097, P117=0.0011527823454086307, P118=0.00045422917873069073, P119=0.0016453653562067002, P120=0.0012499311815356864, P121=0.000208991993475874, P122=0.0003277809957491965, P123=0.0001917864906429228, P124=0.0002500223245609055, P125=0.00034371084333187477, P126=0.00010452174205050012, P127=0.00014072600991371957, P128=0.0002749584275773941, P129=6.41449702665381e-05, P130=0.00014933970706179132, P131=0.00019890230711274156, P132=0.00017680288212568653, P133=0.00022602676229487763, P134=0.00026220557683746354, P135=0.00018177973216148665, P136=0.0003147723947533651, P137=0.00015846708140954317, P138=6.613966233783174e-05, P139=5.8935289059052706e-05, P140=6.073611549146135e-05, P141=9.51231924041173e-05, P142=0.0001506796980360767, P143=0.00018472292143803068, P144=0.0002054073431779438, P145=0.00016323619509734746, P146=0.00013752265375017168, P147=0.00011034673825864171, P148=9.845802091780346e-05, P149=0.0001201267344923324, P150=0.00016324497788883205, P151=0.0002689774914025785, P152=0.0003447346946354829, P153=0.0005615382653957677, P154=0.0009367398529393002, P155=0.0006970117190711619, P156=0.000667373960740277, P157=0.0003490988277576729, P158=0.0006543878541611209, P159=0.0007017101399696576, P160=0.0005769589408377842, P161=0.0004747277492257232, P162=0.00040179465504648035, P163=0.00018814069764475365, P164=0.00012044131959152185, P165=0.00014600509284088447, P166=0.00030847200809697194, P167=0.00040364152206271255, P168=0.000225536049578354, P169=0.0001971439678163754, P170=0.0001677667344361973, P171=0.00014602630680833877, P172=0.00020457830672645597, P173=0.00017810651232182736, P174=0.0003281319938268693, P175=0.0006205513827769057, P176=0.0009259453726912425, P177=0.0007728241035729551, P178=0.0005638751726642875, P179=3.999889270781396e-05),Row( P0=-645.783326264069, P1=59.893634215090955, P2=8.262703564745568, P3=6.32652241186923, P4=-5.87442341682969, P5=-0.09171376502039019, P6=-2.707943634788125, P7=-1.7044203115326493, P8=1.7076043358878692, P9=0.694788919954981, P10=-4.700876921278692, P11=-3.3100429463221346, P12=2.254997217509377, P13=-5.065259273083141, P14=-2.1021220506333655, P15=0.9836413693848888, P16=-0.09855654066695387, P17=0.3448827232172511, P18=1.2025165980608115, P19=2.635580849412331, P20=-0.8290555733742543, P21=1.3911816505903438, P22=0.8080601229306626, P23=1.6979357744328178, P24=0.3700533529921975, P25=4.833850274193238, P26=2.012344744347226, P27=3.4463592150569844, P28=2.4488472687562624, P29=3.135183296417182, P30=3.678642171909022, P31=3.9908269004309793, P32=0.35633835740403025, P33=2.0600142422924304, P34=-0.986877972259358, P35=0.9920281148281611, P36=0.9646262106948871, P37=1.1635613047966293, P38=-1.055710407900122, P39=-1.012878075651735, P40=0.6740205915054548, P41=0.6680934270681208, P42=0.6985089567411175, P43=0.6957724931599, P44=0.722045420538301, P45=0.7227369582689096, P46=0.7215874301977507, P47=0.6890988669039886, P48=0.6720464335865076, P49=0.7087031980231298, P50=0.7169078955219329, P51=0.6957233289858459, P52=0.0004551450316936317, P53=8.599781677676369e-05, P54=0.00012754563263466903, P55=3.902474702764385e-05, P56=7.226645259601307e-05, P57=0.0004523596715784, P58=0.0017397845677948, P59=0.007477803494575742, P60=0.02953017977783581, P61=0.06552031025983111, P62=0.08464554106801889, P63=0.03993415162682016, P64=0.015302559009233316, P65=0.050111987497337564, P66=0.050716176965651366, P67=0.016060592995568227, P68=0.011277276439729717, P69=0.02245477293540505, P70=0.02142085995258799, P71=0.018575874472846976, P72=0.02683163032571525, P73=0.030426632185087694, P74=0.019108398972536043, P75=0.017755468449869194, P76=0.011901335527289975, P77=0.025898813815328697, P78=0.04406383545254519, P79=0.06070290103977639, P80=0.06675467045817111, P81=0.059198084472639134, P82=0.03476422022883969, P83=0.020577122827660426, P84=0.023736806523937864, P85=0.010362315861240761, P86=0.004546004764971178, P87=0.005587189482515922, P88=0.006568649334981796, P89=0.003565934678762728, P90=0.002809564021094129, P91=0.0024808936446700032, P92=0.0022354987816720667, P93=0.002859906734497324, P94=0.011181182565503335, P95=0.03908126330231622, P96=0.015893432378113453, P97=0.0007160449899659656, P98=0.0006013426291380248, P99=0.0005571670779166532, P100=0.0012743028589487307, P101=0.0030469949135996295, P102=0.004963458442663612, P103=0.0009567815960718956, P104=0.00034385374976844674, P105=0.0003304567556920716, P106=0.0009481623229038868, P107=0.004093912219418774, P108=0.010458032420493004, P109=0.0022113246267905623, P110=0.0007465456245237339, P111=0.0021998139344860798, P112=0.0011451053316745192, P113=0.00042609735926278403, P114=0.0003341685324726444, P115=0.0003698507955359878, P116=0.000376845029083141, P117=0.000266006392301068, P118=0.0002595475416615215, P119=0.0003194879512724775, P120=0.00021558649852481635, P121=0.0002634653600483419, P122=0.00014860541672385782, P123=0.0001101018052290123, P124=0.00015663849580047849, P125=0.00014057858491018776, P126=8.830065107502395e-05, P127=0.00013914283806867737, P128=0.0001015750470993869, P129=7.689892008995909e-05, P130=0.00017357713899250976, P131=0.00012610456164112771, P132=0.0001493102827654315, P133=0.0002983932858122045, P134=0.0002619693002099522, P135=0.00020558740365347132, P136=0.00025501206044616597, P137=0.00025826433792076707, P138=0.0003307323559742967, P139=0.0001598178942285129, P140=0.00013683826044619547, P141=0.00011351437859157038, P142=6.391502243172697e-05, P143=0.00013095868741218187, P144=0.00012366897311305854, P145=0.00016133050487738578, P146=0.00011359390182627473, P147=6.424477605380092e-05, P148=3.8633380851373314e-05, P149=2.904627229785755e-05, P150=7.304557179300311e-05, P151=6.591280380057765e-05, P152=6.475173174219775e-05, P153=7.91111646567416e-05, P154=5.992402407067156e-05, P155=6.274497042755825e-05, P156=8.445580043925331e-05, P157=6.475770812332078e-05, P158=7.592781986976262e-05, P159=0.00011562111228360275, P160=0.00011651097405924476, P161=0.00018824613272285824, P162=0.00012392345156367085, P163=6.092484797054737e-05, P164=4.1356118248963276e-05, P165=3.9345644707052e-05, P166=3.0505556716161876e-05, P167=2.594104285384483e-05, P168=1.6839327700623594e-05, P169=1.335098704630086e-05, P170=1.3610624908384532e-05, P171=1.5789561793129938e-05, P172=1.24870661713543e-05, P173=1.3439490546519088e-05, P174=1.1736435438837647e-05, P175=1.3702286230826684e-05, P176=2.15424832860632e-05, P177=2.243102346887425e-05, P178=1.0898670757879476e-05, P179=1.2605227275880594e-06),Row( P0=-741.5639728935448, P1=81.52584050926775, P2=19.306289044207922, P3=16.59511719578674, P4=8.04888117168776, P5=2.1373046707845584, P6=4.675740753842375, P7=5.812103593788255, P8=3.5251345020297196, P9=3.084968291877317, P10=-3.0059490710858405, P11=1.338914538920235, P12=3.0197037699906137, P13=-2.7321928115183685, P14=-3.2174260769125316, P15=3.907324901105267, P16=0.3433981588109222, P17=0.299741603068211, P18=-1.1965766012592982, P19=-0.22576010246233794, P20=-1.2389991002555012, P21=-1.811098688396112, P22=-1.5482668076616968, P23=0.8650772590129142, P24=-1.871144737342065, P25=1.9758925736879374, P26=-2.597994331215484, P27=-0.7375628307713683, P28=-2.4097880363142656, P29=0.8589935864120375, P30=-1.309338735256221, P31=1.304193609332352, P32=-0.8186616321956752, P33=2.928087398902521, P34=-1.1266521416039108, P35=2.0480783492964307, P36=-1.1317334627188644, P37=0.4622256319828216, P38=-1.9502885353603465, P39=1.3199557757363274, P40=0.6904486833213285, P41=0.656352722176103, P42=0.6678039832480543, P43=0.6915987301386364, P44=0.7443284983247112, P45=0.780796172588449, P46=0.8303741900096876, P47=0.7566389330409241, P48=0.6981197949052601, P49=0.6839616990366189, P50=0.6767843702966669, P51=0.6840096540225077, P52=9.357378167821457e-05, P53=2.8213895275401183e-05, P54=3.611678911754821e-05, P55=0.00020734431884558927, P56=0.0020943251023582783, P57=0.006580659238342479, P58=0.0032266193377697354, P59=0.004665115546679089, P60=0.0023232101121538388, P61=0.00078877634929427, P62=0.003059705404235884, P63=0.00433668378398182, P64=0.001143257482171274, P65=0.001470332332401427, P66=0.003957556945697635, P67=0.007364625759863987, P68=0.0031713481393991464, P69=0.0006310802006668507, P70=0.0003572642784885509, P71=0.00016767960324883322, P72=0.00011568109843926072, P73=0.00013272940918119497, P74=0.0006026384753481639, P75=0.0014140558664467381, P76=0.0009865412057515136, P77=0.0003559973815004898, P78=0.0002102262760096078, P79=0.00017970751404105368, P80=0.00021119565208922473, P81=0.0002076471762018655, P82=0.00026077720491234183, P83=0.0002828448544598728, P84=0.00027728660156779147, P85=0.00016412356640176456, P86=8.419205513934673e-05, P87=7.588757601404157e-05, P88=4.9652194736154216e-05, P89=4.5465110824093296e-05, P90=5.394176042446948e-05, P91=9.464593819208794e-05, P92=0.00011160295312281729, P93=3.5758719358965955e-05, P94=6.587745378632443e-05, P95=9.535783850192599e-05, P96=5.941002485448962e-05, P97=3.1335632221280574e-05, P98=4.287795725660309e-05, P99=2.7191534741867378e-05, P100=3.199275923011039e-05, P101=4.7124358809456205e-05, P102=3.884185311453321e-05, P103=3.2896698404040504e-05, P104=1.6969007907163957e-05, P105=1.489336415816578e-05, P106=4.089419584844581e-05, P107=2.9237073623828332e-05, P108=1.92419306041809e-05, P109=2.2277635079247573e-05, P110=1.637279725041286e-05, P111=2.749014857588083e-05, P112=1.9272277775398584e-05, P113=2.0863125064748535e-05, P114=1.6392584804828656e-05, P115=1.826439238964489e-05, P116=1.4853896448920744e-05, P117=1.2447587657561784e-05, P118=1.1950850468839642e-05, P119=1.882235492886311e-05, P120=1.4005896543943474e-05, P121=2.168320046353191e-05, P122=1.7214993130460778e-05, P123=7.962751784055287e-06, P124=1.2353020894650488e-05, P125=8.6620082068086e-06, P126=5.861216731209129e-06, P127=4.910296193657584e-06, P128=4.095284252586164e-06, P129=1.7459060345608262e-06, P130=4.529303346052264e-06, P131=5.162372370209094e-06, P132=3.729541555006451e-06, P133=7.5791925574308215e-06, P134=1.308983388884342e-05, P135=1.605401342882243e-05, P136=1.3430675669587204e-05, P137=8.078592048024246e-06, P138=1.1530305934726423e-05, P139=7.468818479145404e-06, P140=7.785632452419646e-06, P141=1.2144801624993633e-05, P142=1.6982935888803634e-05, P143=2.040828030480679e-05, P144=1.7392741843937075e-05, P145=1.4270724599782711e-05, P146=1.1079506326618045e-05, P147=4.111813311685254e-06, P148=6.330459185831588e-06, P149=6.1516540615839284e-06, P150=1.371485123702963e-05, P151=9.659710232622082e-06, P152=1.213596977207219e-05, P153=1.4944361729659663e-05, P154=1.1974205346747821e-05, P155=9.964549946980277e-06, P156=1.6900707431997464e-05, P157=2.5584494735332892e-05, P158=2.6278562855567447e-05, P159=3.184665093528737e-05, P160=3.680175532815111e-05, P161=2.865992204397533e-05, P162=2.7819719666638185e-05, P163=1.3436513576056189e-05, P164=1.2388604113693453e-05, P165=1.666245223471543e-05, P166=1.0759797121786e-05, P167=2.1342751255662032e-05, P168=1.7204413105379843e-05, P169=1.0151765982951013e-05, P170=5.257775548042597e-06, P171=5.034548082927266e-06, P172=4.756577304341754e-06, P173=6.679331504868362e-06, P174=6.684252808818969e-06, P175=8.009574613412152e-06, P176=1.682922354861945e-05, P177=2.1446148147308685e-05, P178=9.548462883048993e-06, P179=8.327594583359196e-07),Row( P0=-583.7134660248745, P1=55.001130902799005, P2=4.4911888154723005, P3=6.771870637887913, P4=-6.552683090345856, P5=0.029268005977297645, P6=-1.789887519624636, P7=-0.2865606115580729, P8=2.791698164856869, P9=1.8556639057157143, P10=-4.996292242058407, P11=-0.8818487476407927, P12=4.264669901740815, P13=-3.0471809880718985, P14=0.40070822077466883, P15=2.648292992888006, P16=2.063139461406139, P17=3.5874859868687268, P18=4.226158530910867, P19=2.5689087528192283, P20=1.4160546580549682, P21=4.429677518096667, P22=4.98780013358808, P23=6.584073873189393, P24=4.629216410772917, P25=9.102151494387678, P26=5.054402157899781, P27=5.480604123016739, P28=1.8069500587360805, P29=4.969717978253408, P30=3.161955721094419, P31=3.0834366305961978, P32=1.950062780389081, P33=3.77034402698428, P34=1.3203727258288263, P35=3.495905366788414, P36=0.5773484300803265, P37=0.14481879441746406, P38=0.27759578425890025, P39=1.3754780095067165, P40=0.6954801676875444, P41=0.6287306421149048, P42=0.6296686190138464, P43=0.6999863576230652, P44=0.7603491974428443, P45=0.7716009068522344, P46=0.7426752534343015, P47=0.7149415145285851, P48=0.6688311365775994, P49=0.6519849699359239, P50=0.6579409359801465, P51=0.6908133632511132, P52=0.22419172354742387, P53=0.08584555055502804, P54=0.012613505892204576, P55=0.0011254330084944052, P56=0.0008394091007967978, P57=0.0011607757318058782, P58=0.0013050373605158773, P59=0.005059366699527706, P60=0.025630448829311602, P61=0.09283376629906172, P62=0.13898408362369918, P63=0.09381179945894867, P64=0.032135469877270124, P65=0.05401477749518874, P66=0.10725059979871182, P67=0.12727578170185136, P68=0.09607857262629685, P69=0.02568658568059466, P70=0.02001862520639136, P71=0.027070477128935913, P72=0.05001537577426325, P73=0.049872967576791924, P74=0.046173115000669056, P75=0.04816598073973568, P76=0.037076250092692595, P77=0.05665970699132505, P78=0.07558549280188973, P79=0.09832939250019115, P80=0.10193105697992957, P81=0.09107006808190239, P82=0.09588247151523788, P83=0.09266207350606376, P84=0.1364882729944458, P85=0.12806725389756218, P86=0.015256727343585573, P87=0.003590224684550399, P88=0.007904900845188795, P89=0.00523527378950752, P90=0.007129785913028773, P91=0.009242568382427806, P92=0.01753702473423899, P93=0.0063363182699451275, P94=0.012406566518392437, P95=0.009951930781237277, P96=0.007800611000516341, P97=0.018084337605461735, P98=0.005867204465839107, P99=0.002010717919424058, P100=0.000478708870579915, P101=0.0009414744694159168, P102=0.0016796105677991042, P103=0.00322707117532983, P104=0.006316095570481235, P105=0.0012699035350835551, P106=0.00035899313528036877, P107=0.00036615810320301005, P108=0.001303230342304372, P109=0.007341887080183479, P110=0.005766682783338647, P111=0.002480155324004312, P112=0.0016900044423379407, P113=0.00182878232641362, P114=0.0014108685540623724, P115=0.001075238113186037, P116=0.0007238074369922087, P117=0.0013145394489260406, P118=0.000900377887502794, P119=0.00046725942979503504, P120=0.0008209101350928697, P121=0.0007151548631553011, P122=0.0003510672223078044, P123=0.0003201013859816251, P124=0.00023836785546088365, P125=0.00015671257886060914, P126=0.00011071241752862657, P127=0.0003925180665804305, P128=0.00034302535810083154, P129=0.00026521936480710783, P130=0.0009297085034712753, P131=0.0016637628839930852, P132=0.0005402444118295708, P133=0.0015257759316828971, P134=0.0009627973654080781, P135=0.0007317409881634669, P136=0.0012863661094970375, P137=0.0012380684515904802, P138=0.001475906389389642, P139=0.0015042807179122576, P140=0.0007773440373662166, P141=0.0005121083955482297, P142=0.0002461111259175281, P143=0.0002379178063038635, P144=0.0002736151323909754, P145=0.00027579769340724367, P146=0.00034453255596224574, P147=0.000417352862656049, P148=0.00020013215323601226, P149=0.00014152553367667356, P150=0.00010465370253949351, P151=0.00012715284963750543, P152=0.00024323986711238886, P153=0.0002548049912621727, P154=0.00023933357255184455, P155=0.0003391731222363469, P156=0.00031327351657170206, P157=0.0002833830036305316, P158=0.0003568842673526157, P159=0.0005843056987240684, P160=0.0004571131261247035, P161=0.0005744135206435667, P162=0.0006411406557922861, P163=0.0003465770297798888, P164=0.00015857640311346034, P165=0.0002602368992291636, P166=0.00029021054920910896, P167=0.00015934703525511627, P168=0.00012995520336601406, P169=0.00012592900328155728, P170=9.709685748464849e-05, P171=8.429226630082496e-05, P172=9.263712266820571e-05, P173=0.0001273089804959595, P174=0.00018760353268539257, P175=0.00021383777658797056, P176=0.0002698272124389258, P177=0.00023537474441396043, P178=0.00010620039880473847, P179=1.0245778460255671e-05),Row( P0=-479.682960501679, P1=33.23312859208304, P2=-10.696121903025134, P3=1.4520179554687178, P4=-6.606086354311616, P5=-5.968829917395144, P6=-4.138020625895011, P7=-3.8659080889608974, P8=-2.0362065269313483, P9=-0.4884420576650681, P10=-6.508235650405248, P11=-1.182026378587168, P12=2.1002765103589875, P13=2.8971100948348583, P14=8.306799434282482, P15=16.465005978604395, P16=13.752004614891769, P17=12.576089937269723, P18=5.207792035092161, P19=7.4224469751428925, P20=1.3939337388892317, P21=2.4482644949422983, P22=3.248518260630188, P23=2.7968793248826462, P24=-0.5575451948827727, P25=0.5889699232143306, P26=0.6750151449372997, P27=3.14834211229814, P28=1.7412285842682234, P29=4.484595468379881, P30=1.2281252363852966, P31=5.66822094430543, P32=1.5237670147784783, P33=2.540376083764197, P34=0.4260702073551674, P35=0.23163461723920886, P36=0.2998328002896839, P37=0.8710879694432201, P38=-0.14463825139944894, P39=0.3299151798193027, P40=0.7035658764387172, P41=0.6707575790954157, P42=0.6957769174311023, P43=0.6790295238873015, P44=0.630742065651729, P45=0.6200750384961925, P46=0.6490201390810212, P47=0.6857337447533814, P48=0.6189597419997896, P49=0.665533067822004, P50=0.6815278845766768, P51=0.6702276139704064, P52=3.7066596901254503, P53=0.5365302359464422, P54=0.01624007268630926, P55=0.0033254220871867597, P56=0.0012605829722336414, P57=0.0005458152672239076, P58=0.0002380406670866383, P59=0.0007761411349162903, P60=0.006366215645416631, P61=0.01739229188076089, P62=0.030744022864724694, P63=0.04603149871589858, P64=0.05430555039807919, P65=0.1877349531220026, P66=0.9885174742620106, P67=2.221684125350803, P68=1.5552428088543844, P69=1.5696684124389662, P70=1.4527773052803759, P71=0.7272521086250611, P72=0.25076349441284135, P73=0.0722162140653429, P74=0.009003899196407383, P75=0.004972719673719103, P76=0.0052809731634793885, P77=0.017244571696124415, P78=0.03550319686207206, P79=0.05913915142464323, P80=0.07972522371519894, P81=0.1292960068936672, P82=0.35330539677438344, P83=0.6684472891025791, P84=1.2942264361672569, P85=2.19203584638171, P86=1.5913045803133168, P87=0.6743625241606543, P88=0.29632501118231835, P89=0.10338296654736594, P90=0.05862062050557163, P91=0.09012111880684102, P92=0.14836601060818855, P93=0.12937059797104064, P94=0.3560955866072113, P95=1.0641353549405825, P96=0.6497538355212022, P97=0.36591600106587335, P98=0.47752021964492597, P99=0.6245756272985412, P100=2.309699589044883, P101=1.0639236190842911, P102=0.05033832295921071, P103=0.04720259234518679, P104=0.1716154822128503, P105=0.0770244767466502, P106=0.05559818782550709, P107=0.22885151022167446, P108=0.25177531654601104, P109=0.7755061417135815, P110=0.23252124374416294, P111=0.0636773618135315, P112=0.04519403276214082, P113=0.011803525432533189, P114=0.009422687637425828, P115=0.00669236793873459, P116=0.07347074251904911, P117=0.17891509394038435, P118=0.024728347809517222, P119=0.014277364627551638, P120=0.007929425303386152, P121=0.04756950710327431, P122=0.026898045444920218, P123=0.012798479655695489, P124=0.008400895624969651, P125=0.020665214795020426, P126=0.02033368174814098, P127=0.008205850650015525, P128=0.008617816199713147, P129=0.00642234920845494, P130=0.003727367638195253, P131=0.005768977904474857, P132=0.013947042446881342, P133=0.011621909927395577, P134=0.013596247786865495, P135=0.011935056054973633, P136=0.00813024748369822, P137=0.013669009901775366, P138=0.004531249223100352, P139=0.003868539414609968, P140=0.00450327037871954, P141=0.0052326420374640215, P142=0.004630528893670089, P143=0.006489792150939272, P144=0.003826553513398003, P145=0.003734576278212516, P146=0.003977565992136003, P147=0.0018413428014386636, P148=0.0009491156636873326, P149=0.0012603327655676079, P150=0.0019205130081643288, P151=0.0022538242910111104, P152=0.0017576882174324025, P153=0.001917796885666246, P154=0.0015737873109381996, P155=0.002680650709310983, P156=0.002281018847889705, P157=0.003376411255601683, P158=0.0026687319727837972, P159=0.0019006872913554674, P160=0.0009734291008577887, P161=0.0008895794370157976, P162=0.0006303053247396281, P163=0.001162430215990196, P164=0.0006577736995866183, P165=0.0008794038865074975, P166=0.0011484464084501069, P167=0.0008661249232899316, P168=0.0009766694757888518, P169=0.0007042854721132513, P170=0.0008359839164897763, P171=0.0005670303799292875, P172=0.0008351428809191808, P173=0.0010954901606351317, P174=0.0011763035472366421, P175=0.0012133100646782313, P176=0.0009960182110727593, P177=0.0009147515125506212, P178=0.0004085459337609066, P179=3.9325386050781803e-05),Row( P0=-664.1210621145352, P1=64.25140689988379, P2=11.161360654109679, P3=8.24682348470521, P4=1.102149014201261, P5=3.2040779815120572, P6=0.47817156996339066, P7=2.0743881781695617, P8=2.615471021270705, P9=2.891138044355457, P10=-3.5585649357848896, P11=-1.7785998887770647, P12=1.6790245466822313, P13=-3.828209775888892, P14=-2.1537491435358893, P15=-1.1594268828200955, P16=0.29065486426976056, P17=-1.1344386354031883, P18=-2.926852078612385, P19=0.865974147547696, P20=-3.557972428891267, P21=-0.9501849182753749, P22=-1.0520010997299951, P23=1.4895727596344868, P24=-0.1389161376449031, P25=7.07200395762686, P26=5.120699943627834, P27=5.272754476358201, P28=4.451897146501114, P29=5.647955523602361, P30=2.8415169825423723, P31=4.570855540818517, P32=-0.4506201207408655, P33=2.632600106665038, P34=-2.2148837601878997, P35=-0.4000007410200908, P36=0.06584627076249903, P37=0.41810130653525873, P38=-0.5723566673984505, P39=0.1290412690400378, P40=0.691603278543234, P41=0.6993357221536245, P42=0.7860642808335884, P43=0.8000423277337211, P44=0.8107270201094642, P45=0.7886527507301384, P46=0.7145193978927442, P47=0.653664223528652, P48=0.6746257409399544, P49=0.7077114669522261, P50=0.6918898856475655, P51=0.6785490675564866, P52=0.0006761450196156954, P53=0.00012034270476027863, P54=0.00014617477691199226, P55=0.00011535716639778974, P56=0.0003101360917179801, P57=0.000959345918905306, P58=0.0016051163880197819, P59=0.010392511555209172, P60=0.057318332836142454, P61=0.11606769500571976, P62=0.10464967497801547, P63=0.019432114185778133, P64=0.0012130739853750425, P65=0.0009856993639405808, P66=0.002172553889348587, P67=0.0033272269149990995, P68=0.005364700121316319, P69=0.013808643104689683, P70=0.01816774211695023, P71=0.012670185995755324, P72=0.030589630089408716, P73=0.029370762733255576, P74=0.005178746953017401, P75=0.0012458526490563516, P76=0.0014998548280535662, P77=0.006808737992392128, P78=0.01083896627496408, P79=0.015168478601361599, P80=0.016034889306885364, P81=0.012571312147923936, P82=0.006598101311150878, P83=0.006538394429128056, P84=0.005008446682042677, P85=0.003334925480564722, P86=0.0027481318745298353, P87=0.003306373389056605, P88=0.0046609340079322705, P89=0.002793622863939279, P90=0.0021954860808015763, P91=0.004446374578601742, P92=0.007014100131955468, P93=0.00227043276671556, P94=0.002725674287351676, P95=0.004482918190741829, P96=0.00205964974578399, P97=0.00151156842579019, P98=0.003339307058094904, P99=0.002858754464698237, P100=0.0008593557517234124, P101=0.0010532464714692743, P102=0.0012758578977577957, P103=0.0014549447006438026, P104=0.003673746913633434, P105=0.0016390881840167603, P106=0.0005329310913533996, P107=0.00032407904992576113, P108=0.0009178556813792584, P109=0.003289816282188974, P110=0.0014790676740863958, P111=0.0003454707215135, P112=0.00023258386819066242, P113=0.000512328513514753, P114=0.00037388848235554543, P115=0.00020863297664577238, P116=0.00020282422614116888, P117=0.00015509304183196026, P118=6.02061925285845e-05, P119=9.894119638365275e-05, P120=0.00014657931682356907, P121=0.00010329388398877781, P122=8.661691485466644e-05, P123=4.191413638943045e-05, P124=4.233864155144772e-05, P125=2.5756686612523472e-05, P126=4.538794755452638e-05, P127=5.653168968554468e-05, P128=4.853095677074924e-05, P129=4.279863832008931e-05, P130=8.522242677408171e-05, P131=0.00011662067572750019, P132=7.828880084885971e-05, P133=0.0001300106481822386, P134=0.00010246265814981535, P135=0.0001493602070326526, P136=0.00022853077579707823, P137=0.0001576450766920942, P138=9.909182696279249e-05, P139=9.406818234003324e-05, P140=8.069147652961739e-05, P141=6.35904997430561e-05, P142=0.00014856957570521015, P143=0.0003024666472913707, P144=0.00020552612803598065, P145=0.00017934466990769763, P146=9.578019898272302e-05, P147=4.8552729060415946e-05, P148=2.331667880052679e-05, P149=3.2508977676606713e-05, P150=4.221965181531769e-05, P151=4.161552367041045e-05, P152=5.274633661262508e-05, P153=6.678081682862656e-05, P154=6.837263664076131e-05, P155=8.162340401685316e-05, P156=7.274872563312981e-05, P157=7.196958540773156e-05, P158=8.179031205887419e-05, P159=0.00012160790265512293, P160=0.00014036856993040962, P161=0.00017971521102609964, P162=0.0001494733618015255, P163=7.465891945283482e-05, P164=3.368188617190379e-05, P165=2.9209514852740896e-05, P166=3.313616602996806e-05, P167=2.6530390372884322e-05, P168=3.2873850330060944e-05, P169=2.855664984325166e-05, P170=2.85629954373548e-05, P171=4.3020078883875314e-05, P172=2.1962671708803218e-05, P173=2.281548165372565e-05, P174=2.4618234048531655e-05, P175=2.9077051265566177e-05, P176=3.7988221657033415e-05, P177=3.409536324176309e-05, P178=1.7395812193357943e-05, P179=1.76150167633549e-06),Row( P0=-457.79897602536755, P1=25.24147054079365, P2=-14.744828425895596, P3=-5.728896872886563, P4=-8.059045459742837, P5=-5.444719443772753, P6=-8.662370160805967, P7=-4.012703004629664, P8=-0.809917281166809, P9=-1.9217061256013408, P10=-7.027280357711864, P11=-1.9303022416204891, P12=7.1354982087747, P13=9.226968827824448, P14=13.093498265119408, P15=17.29847043751034, P16=12.69748422059043, P17=7.015780437813045, P18=0.15681412910927026, P19=5.641069325223096, P20=1.0758830618523398, P21=4.89057306123123, P22=2.750752440236879, P23=1.944431812331168, P24=-2.1048893758710268, P25=1.8668884091402416, P26=2.5293884775422923, P27=6.517372369489934, P28=3.0375474473671242, P29=4.994007165182126, P30=1.375730178572331, P31=4.7428997119437035, P32=1.4459229852438829, P33=1.8836646930491312, P34=-1.6651841278799135, P35=0.13775124485670176, P36=2.2713132088381185, P37=3.4966606256614408, P38=-0.9044745251371644, P39=-0.03392324377422968, P40=0.6526621102419381, P41=0.6947552461258066, P42=0.7479835641498671, P43=0.697077198114056, P44=0.6587945267108196, P45=0.6880405900319908, P46=0.6801409018318048, P47=0.6043036989102037, P48=0.612038704810278, P49=0.6268518542664289, P50=0.6038254710545558, P51=0.5976893379840172, P52=0.16383884411305744, P53=0.009598390049604921, P54=0.0015556356410216642, P55=0.00027758900089534696, P56=0.0001730733459804601, P57=0.00013495248273172672, P58=0.0004837250754076448, P59=0.003965068847782463, P60=0.007519826942425388, P61=0.013613086300409645, P62=0.030956856397846205, P63=0.052516329384669415, P64=0.07818991920435117, P65=0.23989200956666998, P66=0.41396317335996524, P67=0.4839707929629328, P68=1.2333504932064185, P69=2.7287909547055746, P70=3.7994449207794077, P71=1.8424042160025649, P72=0.48942013552679126, P73=0.2325280468480948, P74=0.08614679837725377, P75=0.029876925016155746, P76=0.013667294400079215, P77=0.02345269309288265, P78=0.04957986428080145, P79=0.08164585941849904, P80=0.0825475107467229, P81=0.07843751817265139, P82=0.1158954970690545, P83=0.18090571741554506, P84=0.45458738487511274, P85=0.9716226153668286, P86=0.9735638578340977, P87=1.7528965524336122, P88=3.683241517956054, P89=1.1776644245587367, P90=0.42331353483117246, P91=0.9767394908000897, P92=3.315948360640913, P93=1.323332839319602, P94=0.4530694675012546, P95=0.3555147228331356, P96=0.2713840731201405, P97=0.7703375112811819, P98=1.6326238013005179, P99=0.6072411364977331, P100=1.173900316929928, P101=1.2987635291807484, P102=0.9177844061579145, P103=0.8951627600060643, P104=1.0044395058140583, P105=1.1332525228993666, P106=0.3342080781955592, P107=0.2323167518241572, P108=0.3259333739007317, P109=0.47586104848875693, P110=0.1757044974258174, P111=0.13448056038441572, P112=0.14032015018837277, P113=0.3012571970882182, P114=0.44475177333854243, P115=0.04791978967758646, P116=0.024346275129657983, P117=0.012902801998416286, P118=0.03892759474177715, P119=0.38517113260052827, P120=0.10368432056704262, P121=0.011489927353892421, P122=0.004975392774826605, P123=0.01359546401825709, P124=0.0259065502897608, P125=0.009110224543383815, P126=0.0034920553277164312, P127=0.010652227446229411, P128=0.016704029909330537, P129=0.0060388016679704375, P130=0.0063924184080044405, P131=0.023663558946116846, P132=0.02881377500346321, P133=0.016514222723904733, P134=0.030745051479507332, P135=0.05971158384942101, P136=0.025803498151967676, P137=0.01537206144380365, P138=0.013918426698038458, P139=0.005800567967495322, P140=0.007588388327167761, P141=0.006173932377845443, P142=0.005805303939727154, P143=0.007346770449269893, P144=0.007869136810964954, P145=0.012259602067904848, P146=0.006143579983174961, P147=0.008111194328662268, P148=0.006257094541291984, P149=0.004434091657664388, P150=0.0043062344338544315, P151=0.0026070549780040676, P152=0.007139771366910991, P153=0.005957368712997542, P154=0.007916973935890147, P155=0.00801921311726868, P156=0.015375582177359829, P157=0.012000670303767752, P158=0.014289232053446383, P159=0.009747413638659615, P160=0.009402364511886585, P161=0.004415356297998671, P162=0.0037687827083227564, P163=0.003490682961142563, P164=0.0025536802905599698, P165=0.0026187360508519997, P166=0.004507907848090639, P167=0.0036254754422933902, P168=0.0034311035657207463, P169=0.002861121767189575, P170=0.00223272353459289, P171=0.00305071418667281, P172=0.004860277725569483, P173=0.005345947921831302, P174=0.005746565746358854, P175=0.00833177428806336, P176=0.008895477402310425, P177=0.004196938674826067, P178=0.001921730189387299, P179=0.00017123184330661235),Row( P0=-530.0096170762484, P1=42.088019652659405, P2=-14.635378738616408, P3=-5.157132676300093, P4=-10.811894402255994, P5=-8.59335042821445, P6=-9.371127899364495, P7=-6.933976510465532, P8=-3.180930616667892, P9=-0.4222128055436114, P10=-6.809310027883155, P11=-5.0639555931252715, P12=-1.3360011946228645, P13=-4.198873472952689, P14=-2.684443420469778, P15=-1.639597983157777, P16=5.268921827330344, P17=6.387632587143262, P18=11.134150162043579, P19=14.050467884324329, P20=6.25697542261187, P21=8.035054744142835, P22=1.2780767497105117, P23=3.1285040646550217, P24=-0.08663585727929209, P25=3.7211124215210063, P26=0.15771787391051106, P27=2.9181840262985723, P28=-2.5347053207172334, P29=-1.671246300106729, P30=-1.4797897249411087, P31=2.5029408763546703, P32=-0.5822486664067648, P33=2.1121303478287734, P34=-2.0625514040525297, P35=0.28558840761122783, P36=-0.818578229774135, P37=1.4489830654541427, P38=-0.5883464584956107, P39=1.062481119022183, P40=0.6231840364515574, P41=0.5847341591794594, P42=0.6073718273627288, P43=0.6254390816982447, P44=0.6456063138956494, P45=0.6451113067059011, P46=0.6660672174191657, P47=0.6272590261651944, P48=0.6146787759402677, P49=0.671761696819425, P50=0.6124260419654893, P51=0.5859941160279197, P52=0.00024663367363856486, P53=0.00012129795050863762, P54=7.66077218857236e-05, P55=5.782556158240621e-05, P56=8.23817627356637e-05, P57=0.0002963425779003392, P58=0.0007071268724772507, P59=0.002808960521950963, P60=0.010056400327785707, P61=0.037001432234528106, P62=0.15148702844429265, P63=0.19097714569142973, P64=0.2759795249795849, P65=0.5155102763134412, P66=0.8232136321127062, P67=1.4405485615623452, P68=0.5111832468054412, P69=0.019729276808210898, P70=0.007362200610006374, P71=0.005214468253176577, P72=0.007568504896494961, P73=0.011323098144929034, P74=0.02096036860546793, P75=0.03234743875209441, P76=0.06038518356765312, P77=0.502407445196781, P78=1.496313833045975, P79=0.9844616681811535, P80=0.19878197981920576, P81=0.12393643411199147, P82=0.19012859751744074, P83=0.3937803909183207, P84=0.2809604937129442, P85=0.08482344096594127, P86=0.023957575506722065, P87=0.04991955063506966, P88=0.11695341071580508, P89=0.1818303854256861, P90=0.4418349343948425, P91=0.7767441437754017, P92=0.744878586879511, P93=0.4509602458056553, P94=0.4197312494994875, P95=0.12815386866038184, P96=0.0322848002827644, P97=0.013361230336777325, P98=0.020713500088403936, P99=0.06580791063357362, P100=0.3231906501892679, P101=0.5432502175537265, P102=0.16717644804618126, P103=0.021477060740048633, P104=0.01477886160794329, P105=0.04175833645327342, P106=0.032248896365292386, P107=0.10297899203019528, P108=0.1771962948926242, P109=0.20390085086404058, P110=0.0575844641039128, P111=0.0326227584657644, P112=0.034605264940048334, P113=0.04730923760955351, P114=0.0491577300760638, P115=0.024261019212971968, P116=0.005596054458693257, P117=0.023856023785086018, P118=0.01904994909841349, P119=0.003935050558842902, P120=0.0033505779418356476, P121=0.020171976409877876, P122=0.003951046456089315, P123=0.0009142082283471352, P124=0.0018372994239699346, P125=0.004543112787578712, P126=0.006968120541375165, P127=0.0034216318751458816, P128=0.001626489438183648, P129=0.003414060005506942, P130=0.010717688353456102, P131=0.005636912484508689, P132=0.0025986393727214204, P133=0.00759233222601523, P134=0.0063134005483470755, P135=0.004660714951315419, P136=0.00917613381886885, P137=0.0023987373626743937, P138=0.0025041946149136234, P139=0.002471483376084449, P140=0.0015349118771669835, P141=0.0006733525331216836, P142=0.0006809580894628836, P143=0.0007467097648183034, P144=0.0006875780033183654, P145=0.000617296582610769, P146=0.0007908496746349969, P147=0.0008287134423432335, P148=0.0004025808846188484, P149=0.0002751446105903268, P150=0.00024260464392936656, P151=0.0002500163313569822, P152=0.0003024074508526248, P153=0.0003790768567959507, P154=0.0005813171832758018, P155=0.0010873392601458245, P156=0.0017383030241294123, P157=0.0016118736455419105, P158=0.0013960551791024707, P159=0.0010267290019515835, P160=0.0013631922118884736, P161=0.0021872174687758155, P162=0.002132937509456968, P163=0.0020540778582428397, P164=0.0015985671672432632, P165=0.0012037326324355415, P166=0.0010300776266809765, P167=0.001359087114562128, P168=0.0010884945281990973, P169=0.0007055966846576889, P170=0.0005149768070365225, P171=0.0005970601303816495, P172=0.0007400784843200949, P173=0.000997769050014982, P174=0.0013645661387043742, P175=0.0016373531010386475, P176=0.0016157447205632951, P177=0.0011235794895253848, P178=0.0006077357802831021, P179=7.378811396689323e-05),Row( P0=-612.3571937890658, P1=67.1047371057403, P2=3.1621393714964414, P3=-4.506976910015197, P4=-7.580783630888477, P5=-1.2587147189310164, P6=-6.577418477896306, P7=-4.987081578212462, P8=-0.3356658686954142, P9=-2.3401077693640944, P10=-9.169979057055512, P11=-7.408896900485357, P12=0.660823447579382, P13=-5.618810423609986, P14=-2.976666909915519, P15=-1.1260256310460826, P16=-3.276837493489546, P17=-2.7943432068378367, P18=-0.3430346021475417, P19=-0.9163362584613385, P20=-3.6444099847951636, P21=-3.158880020798937, P22=0.9306662076944496, P23=1.1991463307666783, P24=0.8442705641251423, P25=7.500870404928355, P26=4.968536824734902, P27=8.41067706427501, P28=5.401039898259592, P29=4.477998769011363, P30=2.085662193482715, P31=2.802634483587967, P32=-1.4975201438499446, P33=3.361545143802784, P34=-0.6699040494579487, P35=1.698351930857808, P36=-1.05461857955577, P37=-0.1441601865798994, P38=-1.7946350408208913, P39=-0.33952162056987756, P40=0.6332508021919777, P41=0.6115174258925071, P42=0.717629628052185, P43=0.7761177049763526, P44=0.7419097903858164, P45=0.6905558857472945, P46=0.6496868691050418, P47=0.6236578002157417, P48=0.59783563405728, P49=0.6124104748931103, P50=0.6103191219300182, P51=0.6162754447352753, P52=2.8789753903482e-05, P53=1.5645238924469663e-05, P54=3.126587811870504e-05, P55=3.1349032583749935e-05, P56=0.00017160376695903135, P57=0.001027988983258657, P58=0.0022885649905560225, P59=0.011229731238405066, P60=0.06444955020833826, P61=0.1403115480368283, P62=0.11666062420597839, P63=0.055735133496417844, P64=0.01700493294505326, P65=0.006489599371462615, P66=0.010130744250041316, P67=0.018150019568355175, P68=0.017935800166927193, P69=0.03903477428771139, P70=0.0894778585665514, P71=0.07718962659434397, P72=0.05518651343537033, P73=0.05259490877975186, P74=0.026770604013505176, P75=0.009905035525228105, P76=0.011403848408038667, P77=0.029564568441569072, P78=0.029504530744846835, P79=0.03797783011697001, P80=0.051601540018133364, P81=0.03782476236106599, P82=0.03645923257463642, P83=0.040324454281350015, P84=0.03655541896199825, P85=0.0236847143551361, P86=0.008116471015805139, P87=0.013036099674532255, P88=0.03619133548273918, P89=0.04645929475472178, P90=0.03749015331589941, P91=0.0331969352576851, P92=0.04926173129556578, P93=0.061261480988418306, P94=0.04821029199638811, P95=0.018833991179959965, P96=0.01620115360487741, P97=0.024899055675014187, P98=0.028181402778583897, P99=0.01705171146734218, P100=0.034628610992719744, P101=0.017201230062420268, P102=0.002288227603609891, P103=0.0014723193995640992, P104=0.004061070379879191, P105=0.020305892980495576, P106=0.03106080600058954, P107=0.00677684300667738, P108=0.0039602153717308575, P109=0.004304725367463797, P110=0.008690447517877706, P111=0.017286606939016624, P112=0.004172870947682689, P113=0.0014109162677921644, P114=0.002698277328916653, P115=0.002338985408028376, P116=0.0006620391248878425, P117=0.001405819497085375, P118=0.0012801875980598442, P119=0.0009229698099832339, P120=0.0018153469994043612, P121=0.0005598926553161778, P122=0.0002932248686316351, P123=0.0002332221705942693, P124=0.00013041983669863496, P125=0.00015119287934113333, P126=4.901940902429386e-05, P127=6.758272527174822e-05, P128=8.8349773817742e-05, P129=3.430961828836237e-05, P130=5.79083380359151e-05, P131=7.65799582846833e-05, P132=0.00010697717003791837, P133=0.0003241107237662573, P134=0.0003730331625720175, P135=0.00024040594617375096, P136=0.00024242584104916648, P137=0.00021519375143357473, P138=0.0002643122954124935, P139=0.0001821596574477414, P140=0.00013380072909539403, P141=0.0001270208133387757, P142=8.239494801906413e-05, P143=6.705941931584077e-05, P144=8.347386539406224e-05, P145=0.00013999443548322557, P146=0.0001250553867124411, P147=8.921215566102057e-05, P148=4.53925421041768e-05, P149=3.994566782440664e-05, P150=6.474016189647006e-05, P151=0.00010082346405377848, P152=0.00024412165532838517, P153=0.0004232291813461687, P154=0.0006402136526247603, P155=0.0009313274183585708, P156=0.0012339490071111797, P157=0.0009613732676177465, P158=0.0007376083166431696, P159=0.0010721243762538645, P160=0.0012050785256553834, P161=0.001075620291246878, P162=0.0007453504250111467, P163=0.0004034088584553197, P164=0.0003267841733368765, P165=0.0003581213319580603, P166=0.0003757923846267313, P167=0.0003539561518829336, P168=0.00028768209319669243, P169=0.0004023825417913644, P170=0.00019575748598170875, P171=0.00022390294575559935, P172=0.00022991435442888032, P173=0.00022195687738077701, P174=0.0003289931902287162, P175=0.0004855303603373773, P176=0.0005136801210755382, P177=0.0005148424031332578, P178=0.00034491952300713907, P179=2.944213689127314e-05),Row( P0=-609.9730822255964, P1=54.76968760680045, P2=4.657599221220618, P3=6.186587460203875, P4=-6.34179173869906, P5=-2.5495527059751257, P6=-1.7936264482039155, P7=-3.9012973551983716, P8=1.9427523917799534, P9=0.09275462693877592, P10=-3.273815552308765, P11=-1.9353067321219852, P12=1.5444506103731002, P13=-4.082528533651809, P14=-1.0430884147191901, P15=0.4037730724802253, P16=1.597786973097649, P17=1.5704505415328787, P18=1.317852415789303, P19=2.798598415939075, P20=-0.7827525180880424, P21=1.3077281631025626, P22=3.2096553051520353, P23=5.075341746846876, P24=2.7466383423440943, P25=7.494181757142903, P26=4.765620394530222, P27=3.615204374154615, P28=3.191794332799549, P29=2.8725433046414532, P30=3.765830877507764, P31=5.07049987976057, P32=1.1098189462788413, P33=2.5290377148106407, P34=-1.4098851910216859, P35=1.8876106233784473, P36=1.0284607391667107, P37=1.7907055751299823, P38=-0.9813424358570054, P39=-0.8799762438432212, P40=0.6918963195405391, P41=0.6750565922393126, P42=0.7196613581402134, P43=0.7243995736023666, P44=0.7230065601864206, P45=0.741114708746112, P46=0.7387601259481378, P47=0.7000271220797083, P48=0.6561262854499628, P49=0.6802898132411159, P50=0.6935532700470821, P51=0.6905254687746248, P52=0.23093817178491036, P53=0.042074335491939396, P54=0.00510242725210707, P55=0.0006318266549821309, P56=0.0002462015361452973, P57=0.00048427892259817113, P58=0.0012755969413024296, P59=0.00768297793488866, P60=0.03340986398237121, P61=0.07039025246765064, P62=0.14150514322504362, P63=0.09834807531070602, P64=0.018163649252464425, P65=0.04816603083748453, P66=0.10203913521623972, P67=0.048131124640780495, P68=0.013058032516285211, P69=0.02018329022017599, P70=0.023075008738789565, P71=0.01585226358195121, P72=0.02887629458621407, P73=0.04884121744308864, P74=0.03806484728308245, P75=0.01955681904902603, P76=0.014716210603148422, P77=0.03560584081211381, P78=0.05897607099446464, P79=0.07841185925495459, P80=0.0713883056971779, P81=0.0694765922657387, P82=0.0641692387954238, P83=0.050785762228378745, P84=0.06268390611171551, P85=0.042871259714051715, P86=0.008002233680850703, P87=0.003076805065896965, P88=0.0030759474430832588, P89=0.003268870271534485, P90=0.0068615420622581596, P91=0.006815898174734553, P92=0.004484555933889308, P93=0.0031863870499301544, P94=0.007151542357587853, P95=0.02321025086793507, P96=0.020137296425318488, P97=0.005005306616450654, P98=0.002211986659323431, P99=0.0008147865244738489, P100=0.0014399581761517, P101=0.002132793922288118, P102=0.0029695538902297, P103=0.0019417579583729107, P104=0.00029248416856434805, P105=0.0004516896184931612, P106=0.0010533711856559875, P107=0.0020177180478765386, P108=0.01278091804579468, P109=0.009762042233346431, P110=0.0011251561535989853, P111=0.0017141298476320737, P112=0.0017491181901829133, P113=0.0009009597920226592, P114=0.00044077038522444213, P115=0.0012641061400474336, P116=0.0009346353010817237, P117=0.0003862258637781937, P118=0.0006676222071742338, P119=0.000532719388070465, P120=0.0002375508173566792, P121=0.00029302055039603074, P122=0.0002097366281545394, P123=9.412555812274466e-05, P124=0.0001231181860601695, P125=7.766935819284781e-05, P126=2.878500764642286e-05, P127=9.989795350162952e-05, P128=0.00018139380444076405, P129=8.667018131456545e-05, P130=0.0003934042141150435, P131=0.0012598363471166844, P132=0.0002600907321002477, P133=0.0003663876639757302, P134=0.0007403567374528976, P135=0.0003224187427015943, P136=0.0006173618996723425, P137=0.0006353072854704938, P138=0.0003154851681350251, P139=0.0008156988226354123, P140=0.0003187930380851936, P141=0.00045874947102084705, P142=0.00039064286511262815, P143=0.0004621076045840848, P144=0.00033128008581804585, P145=0.0002559197283985342, P146=0.00020277141485830637, P147=0.00020861039558750757, P148=0.00013114388202263268, P149=7.560268649738772e-05, P150=6.595557936730134e-05, P151=6.325953344397822e-05, P152=8.193425193594593e-05, P153=0.00012068233804273448, P154=9.863255215724807e-05, P155=8.794048557212419e-05, P156=8.065145070366209e-05, P157=5.53577612471381e-05, P158=0.00013538482472229678, P159=0.000222055136455144, P160=0.00016682444440807012, P161=0.00020060837702100433, P162=0.00021788929138790138, P163=9.173498804436124e-05, P164=6.632458799074522e-05, P165=4.565853745131012e-05, P166=4.146464133037481e-05, P167=2.9762882468016857e-05, P168=3.288585550570899e-05, P169=3.332506659374685e-05, P170=2.2152172859473538e-05, P171=2.1639532140716173e-05, P172=2.2516529998243422e-05, P173=1.9268284819640555e-05, P174=1.7703884893133428e-05, P175=2.8233590516003865e-05, P176=4.296017722462358e-05, P177=5.5636269981765765e-05, P178=4.1198729571398474e-05, P179=3.2705576325662366e-06),Row( P0=-725.49598039553, P1=75.47220866089724, P2=19.81702545431598, P3=11.663593930813228, P4=0.0124944623905709, P5=2.6432249197126594, P6=-0.3078460471499355, P7=4.282432072711298, P8=4.085908102053697, P9=-1.019462249546065, P10=-2.459837553475934, P11=0.8042895659938423, P12=1.6380066879652067, P13=-3.085125803801074, P14=-1.9957756362370245, P15=1.4801168009232968, P16=0.6046151186285823, P17=-0.4116781462841662, P18=-1.8467437608318618, P19=0.2677596355002361, P20=-3.395279970801837, P21=-3.877194996124206, P22=-3.638873107424203, P23=-2.3123408356894886, P24=-3.663639179071561, P25=0.5917996736881269, P26=-1.3256765574265732, P27=0.5074587656609475, P28=1.458804613378518, P29=3.5748071136739927, P30=2.428873148825639, P31=4.483744651781346, P32=-0.711057830890145, P33=1.517993671690875, P34=-2.4656110288078033, P35=1.4737717812293902, P36=0.14950012418973477, P37=2.288747992939427, P38=2.090723251815537, P39=5.052600761409843, P40=0.6648727102412589, P41=0.6899998468211985, P42=0.6680318760766527, P43=0.7106434421523679, P44=0.7378858278816236, P45=0.7579873834032015, P46=0.7269294173514813, P47=0.7142805288678961, P48=0.7563812574274261, P49=0.7616591773217816, P50=0.7092353365923301, P51=0.6544063781370331, P52=4.845515989184627e-05, P53=1.3765575424517805e-05, P54=1.727312081379038e-05, P55=9.293957127481519e-05, P56=0.0015489391605901, P57=0.006872725434488835, P58=0.006483988859177514, P59=0.007582378430737375, P60=0.011754899431869106, P61=0.004110938522895311, P62=0.0021199655689432472, P63=0.0028431466331794103, P64=0.0059086792964544825, P65=0.013577196062249774, P66=0.006217944879852662, P67=0.0032277024061341877, P68=0.0109697691331099, P69=0.008800926157325929, P70=0.007602874226103207, P71=0.0035990124661455466, P72=0.002191824475511477, P73=0.001517105511768826, P74=0.0006327979491797001, P75=0.0005174230945651178, P76=0.0013379790722781453, P77=0.004704384879499709, P78=0.0041802279833387775, P79=0.0075953351331381974, P80=0.0066372814799175, P81=0.0023519128725007967, P82=0.001366509481250439, P83=0.0008572147403808259, P84=0.001743128455447182, P85=0.005255602546294062, P86=0.0029072923287107835, P87=0.0005459469487291517, P88=0.0006765543044493439, P89=0.0005321674221043103, P90=0.000396063242826475, P91=0.001479545932579283, P92=0.010117575892572781, P93=0.0026630158167371847, P94=0.0010430778944699709, P95=0.0008585542813226409, P96=0.00026825335846584954, P97=0.004497979067808888, P98=0.007239003495688151, P99=0.0007737435766476214, P100=5.281501231191661e-05, P101=0.00015781805830100074, P102=0.0017537769301006543, P103=0.0014316645146034786, P104=4.139762376256621e-05, P105=5.514633923445371e-05, P106=0.0004961532431356453, P107=0.00041905346948932964, P108=6.429307986411698e-05, P109=0.00022941587583383117, P110=0.0021473486568027254, P111=0.0011199914089656893, P112=4.311816404060039e-05, P113=0.00043664242523885416, P114=0.0002070720743227314, P115=3.445060115516735e-05, P116=0.00015772011222459745, P117=5.926769037068057e-05, P118=5.3410813466775255e-05, P119=6.521997886741006e-05, P120=3.374211248793236e-05, P121=1.7900452726241906e-05, P122=2.1969175360311676e-05, P123=1.2820684112723714e-05, P124=1.689874328468303e-05, P125=1.5416656875842358e-05, P126=3.8109253335636374e-06, P127=6.4674448872721635e-06, P128=1.5384738625475392e-05, P129=5.086207519797636e-06, P130=9.762199523149846e-06, P131=1.2053579427762872e-05, P132=1.2585188964975873e-05, P133=1.6589175644975886e-05, P134=1.6192201877929704e-05, P135=1.068261224484505e-05, P136=1.094570318015069e-05, P137=9.074471026112123e-06, P138=8.998712042004068e-06, P139=1.0827071457394082e-05, P140=1.8207022851648566e-05, P141=3.2647036990287414e-05, P142=5.923537083031189e-05, P143=0.00011524338938871685, P144=9.750402908196883e-05, P145=4.6940635504009654e-05, P146=2.38014983660675e-05, P147=3.0164166067473325e-05, P148=4.7644239181177176e-05, P149=6.266774120865926e-05, P150=9.530661378245027e-05, P151=0.00010723339059464247, P152=0.00019302269691590676, P153=0.0002397989800155347, P154=0.00024907201141306014, P155=0.0002918807078119437, P156=0.00022880485108348313, P157=0.0002279555539263642, P158=0.0001456275311015853, P159=0.00015000137152595738, P160=0.00014962378454555196, P161=0.00010636333946712323, P162=8.804530025089541e-05, P163=8.649887032588949e-05, P164=0.00011410990464810303, P165=5.460547225939001e-05, P166=4.920225780870263e-05, P167=6.296686174301782e-05, P168=2.9654299601629287e-05, P169=2.6999465029185214e-05, P170=1.6793017502804998e-05, P171=2.1083405000134105e-05, P172=1.8448370607995573e-05, P173=3.025495450064733e-05, P174=3.2721395920712225e-05, P175=4.7107777815348345e-05, P176=3.953185781813525e-05, P177=4.145762188087279e-05, P178=1.4463545618095498e-05, P179=1.5974122147908905e-06),Row( P0=-646.6186356755658, P1=61.12029657854691, P2=5.997715869425313, P3=0.5652545306511964, P4=-3.263400519562612, P5=1.688789174996938, P6=-3.9023647779913, P7=-0.3684275196912428, P8=-0.19873615358173485, P9=-0.5123126303272638, P10=-2.6131917252484502, P11=-4.086052303607767, P12=1.3078260233470007, P13=-6.011992228094592, P14=-1.20323488118865, P15=-0.0648740629145801, P16=-0.13193241916082052, P17=-2.0285489282632008, P18=0.2746621930645256, P19=0.48456042416546696, P20=-1.589884717664211, P21=-1.712582635347511, P22=0.8867846618280096, P23=2.197081207410648, P24=0.6423493692560529, P25=6.780365589892428, P26=1.8299011634737956, P27=3.6456584078349685, P28=3.398136698695208, P29=3.8309818672822438, P30=2.591897650824633, P31=4.796622548682543, P32=1.3822323539479322, P33=1.7378811240722143, P34=-0.7616331577722509, P35=1.0582447888700217, P36=0.5442315802714601, P37=0.5789482217171102, P38=-0.8430835582808731, P39=-0.3228454567870431, P40=0.6949174854920173, P41=0.689798260663627, P42=0.7224699536076166, P43=0.7218198190001253, P44=0.7287117174551144, P45=0.7296437680775062, P46=0.6922155909109127, P47=0.6655457223873049, P48=0.6233604824280079, P49=0.657467878424116, P50=0.6993111078315011, P51=0.6989514324131405, P52=4.538622712281109e-05, P53=4.1532141290183995e-05, P54=5.747822021037507e-05, P55=2.939105884180147e-05, P56=0.00020380311063219371, P57=0.0010877330666310273, P58=0.0030995480290153627, P59=0.011742449957431894, P60=0.04872740623313257, P61=0.07853522380081479, P62=0.07058136228323279, P63=0.04209477965586279, P64=0.0050097972972843975, P65=0.004430602496924063, P66=0.009418492861425853, P67=0.01219650928343005, P68=0.013313366279410257, P69=0.02516065209068446, P70=0.027253725723762206, P71=0.01983147506719861, P72=0.01482762319843266, P73=0.011591800753708673, P74=0.00933071009425108, P75=0.010297161996098179, P76=0.004044860510682003, P77=0.005242554800524089, P78=0.016609781258126982, P79=0.030394190021794595, P80=0.027077783239918295, P81=0.027292032953857178, P82=0.02464742848601085, P83=0.013335086505054168, P84=0.014019796215878741, P85=0.014357933696369755, P86=0.009052378546714595, P87=0.00887092871160554, P88=0.008523596238021505, P89=0.00409439178186808, P90=0.005662191002661446, P91=0.015963030291667502, P92=0.029985016920019737, P93=0.028316455848424762, P94=0.056020025701143346, P95=0.10783058676232088, P96=0.02653775164842402, P97=0.0011941953460906502, P98=0.00581974782269914, P99=0.008789974395128415, P100=0.01767162037699847, P101=0.04225161893662459, P102=0.058477617503653774, P103=0.010598490331512507, P104=0.0012605961710308506, P105=0.0026838551645725736, P106=0.00577280751470667, P107=0.009115116100310544, P108=0.00960577013379159, P109=0.0033579952099160306, P110=0.0018838114728490082, P111=0.001486050444620228, P112=0.0008515999969806278, P113=0.0006962281725057473, P114=0.0003072688109477691, P115=0.00043459471670904016, P116=0.0008186474012429803, P117=0.00045848895596788526, P118=0.0005126102340658612, P119=0.0003332804243398123, P120=0.00023080281360469209, P121=0.00011703413096629242, P122=0.00010917262280739931, P123=8.300150526644728e-05, P124=0.00010049803399552674, P125=7.412265154167934e-05, P126=2.378124496944522e-05, P127=9.094489409209226e-05, P128=7.304868924788394e-05, P129=2.136607273081348e-05, P130=3.2211709052619814e-05, P131=5.580990815095899e-05, P132=6.30353340286569e-05, P133=0.0001383117533291372, P134=0.00012011069309428367, P135=8.222304133211228e-05, P136=5.966910877169329e-05, P137=5.824704300474252e-05, P138=6.418228182716014e-05, P139=3.06821289875501e-05, P140=2.6297209402787966e-05, P141=6.373328568378175e-05, P142=6.016738103249264e-05, P143=7.441744977534565e-05, P144=0.00012843260905685103, P145=0.00015382495258086168, P146=4.1798307682008596e-05, P147=2.6913926082410273e-05, P148=2.5854565623964566e-05, P149=2.7797088439780257e-05, P150=6.465247427828055e-05, P151=9.693001102896772e-05, P152=0.00011695668546249918, P153=0.00018347075302557162, P154=0.00021548982662217473, P155=0.0001906477782559738, P156=0.00032563380208023933, P157=0.00027263281202525647, P158=0.0002602857832601526, P159=0.0007182779245303927, P160=0.00068165970105912, P161=0.0007014406017116707, P162=0.00037444023624399347, P163=0.0003690330871955315, P164=0.000398752674703277, P165=0.0001703355592886916, P166=0.00019870643322857226, P167=0.00018277132160905214, P168=0.00030874092825134927, P169=0.00016715906237178054, P170=6.050539254336509e-05, P171=0.00012195013365260012, P172=0.0001752103626880345, P173=0.00014975723494850494, P174=0.0001045276001521154, P175=7.797123092458199e-05, P176=0.00012311457148268213, P177=0.0001737406615875838, P178=7.033703115689506e-05, P179=5.221201426520706e-06),Row( P0=-738.9450269089174, P1=76.17376407921523, P2=24.046003733289776, P3=16.959731097931424, P4=6.526299950876896, P5=4.674064985213401, P6=0.5199653891529435, P7=6.372855908697974, P8=0.8346511840947235, P9=1.9826147978535515, P10=-6.0745530195708906, P11=-0.6836615567308718, P12=2.2828546737106805, P13=-3.184186317671673, P14=-3.8394431820036155, P15=2.232752358717341, P16=1.4669633277481984, P17=-1.2076774639290437, P18=-2.421512876931986, P19=-0.46161698653725575, P20=-1.265232382255106, P21=-2.48344879572576, P22=-2.513985254685002, P23=1.1286076970739838, P24=-4.561753820129304, P25=2.2531969284271565, P26=-3.5598047573303258, P27=-1.3069167646994455, P28=-1.1096980215816932, P29=1.104400155181599, P30=-1.7893828645030188, P31=3.4068024815880373, P32=-0.6515650782421737, P33=4.455923747513983, P34=1.3416661404398669, P35=4.738350665376157, P36=2.0705583020789824, P37=3.184708497134881, P38=1.4410116500287604, P39=3.705331091903256, P40=0.5977202441504215, P41=0.5722783034091077, P42=0.6351313337877017, P43=0.6668617802765644, P44=0.6830987726811761, P45=0.7026763966115641, P46=0.7125113945785121, P47=0.6637203033792233, P48=0.6827581911026589, P49=0.6977732675222793, P50=0.6988151475114274, P51=0.6617294986418037, P52=6.715286795475018e-05, P53=9.30100199690936e-06, P54=1.3793526249545017e-05, P55=5.937186184546216e-05, P56=0.0014863515997746075, P57=0.007843461194844721, P58=0.008185585520376848, P59=0.007391972147409634, P60=0.0017591387861241833, P61=0.0005517157250376611, P62=0.002192464753702268, P63=0.0029612711214303165, P64=0.0022602728995158934, P65=0.004993914535764505, P66=0.011136851792791277, P67=0.011776748998546412, P68=0.002754246035920238, P69=0.000386868074940231, P70=0.0003119610041357367, P71=0.00027394880960858985, P72=0.00040518570246957173, P73=0.0005375910523171305, P74=0.0011227140888538983, P75=0.0010736939010113525, P76=0.00029374557937104066, P77=0.0001678595386502214, P78=0.0002950913894288961, P79=0.0004951356597698746, P80=0.0005722626094938442, P81=0.0008211832722626482, P82=0.0010547930194744776, P83=0.0004859255878110656, P84=0.0002279769527949195, P85=0.00031351985121241523, P86=0.00012581411630748214, P87=7.431273665583349e-05, P88=0.0003060632077917917, P89=0.00022697481330088246, P90=0.00011244193881829165, P91=0.0001365109227285565, P92=0.00013547685027982063, P93=0.0001804031145330882, P94=0.0007089638332884611, P95=0.00030472486107845233, P96=5.6591311110495134e-05, P97=6.580205342788555e-05, P98=8.037618891491258e-05, P99=6.280380206720096e-05, P100=2.7687073444996068e-05, P101=3.966762291519735e-05, P102=6.750619221698894e-05, P103=0.00010967880775794463, P104=2.8723679251369926e-05, P105=2.1158102483327857e-05, P106=9.736675826621279e-05, P107=4.8557184438767816e-05, P108=2.35710990904031e-05, P109=7.574805425813974e-05, P110=5.7159212249919205e-05, P111=3.7573428691097866e-05, P112=2.117481179168876e-05, P113=2.6504491740696626e-05, P114=2.9777777155016423e-05, P115=3.58772194088718e-05, P116=2.7557054124893136e-05, P117=2.1398720373544756e-05, P118=2.627316334105923e-05, P119=2.405170794220841e-05, P120=1.4144628837959928e-05, P121=1.3247747971893929e-05, P122=1.6556416124511143e-05, P123=7.23642394230911e-06, P124=6.867023363723435e-06, P125=6.764485385320815e-06, P126=2.8722487532770983e-06, P127=3.995547114344803e-06, P128=3.488077459608662e-06, P129=1.999822942178423e-06, P130=4.127284488742672e-06, P131=6.936910295581905e-06, P132=8.208619653404973e-06, P133=1.3917414440628284e-05, P134=1.4560122738889711e-05, P135=7.742051521414478e-06, P136=6.223621344591895e-06, P137=5.145902052285368e-06, P138=6.416841174615144e-06, P139=8.19803981297741e-06, P140=1.6254223024695622e-05, P141=3.047633573610042e-05, P142=2.54025894019971e-05, P143=3.4392605581785055e-05, P144=4.499102264434816e-05, P145=1.3470061155834972e-05, P146=3.5256212172883397e-06, P147=5.68258267445329e-06, P148=7.093043470343276e-06, P149=1.0557153353926455e-05, P150=2.148204033997495e-05, P151=3.567930484624723e-05, P152=0.00012211785327761283, P153=0.00010902664302861074, P154=8.66057850702122e-05, P155=4.0104394563940305e-05, P156=4.821221967822647e-05, P157=5.8319642122498763e-05, P158=5.262626183379109e-05, P159=7.817611881095548e-05, P160=6.954108602309062e-05, P161=5.094603161835723e-05, P162=4.895843915500416e-05, P163=3.453230442087886e-05, P164=3.8702372946519075e-05, P165=4.613837841893622e-05, P166=3.438637638155521e-05, P167=4.057349373152021e-05, P168=1.7199037801408667e-05, P169=1.5838216251976402e-05, P170=1.3045301140702813e-05, P171=1.5157484705830222e-05, P172=1.281921681695112e-05, P173=1.5434937676082418e-05, P174=2.216688539345957e-05, P175=3.3411119225555626e-05, P176=2.6988368343537776e-05, P177=2.461436361880793e-05, P178=7.400684769859163e-06, P179=5.466567133937947e-07),Row( P0=-753.3515673638954, P1=77.61692876391072, P2=24.490075778602865, P3=19.969241581393362, P4=8.756458796105044, P5=5.793496174136248, P6=3.5944999067210195, P7=4.201872300165031, P8=5.116081767803535, P9=3.1623083166335344, P10=-2.636959574542812, P11=1.9291765015697435, P12=0.9353955868080324, P13=-3.34865625902378, P14=-2.910734924857888, P15=2.7050551613982394, P16=-0.7978862028980284, P17=-1.5020045449158037, P18=-0.7793193703644197, P19=0.8439065629726874, P20=-0.824639952199308, P21=-1.7012550088638256, P22=-1.709452381843976, P23=-0.888135653766597, P24=-3.7846130256179547, P25=2.1940116005451, P26=-2.8040224538090186, P27=0.2580332218837057, P28=0.7702831389217751, P29=2.922066409264375, P30=-0.0832975412801473, P31=3.1533176386444266, P32=-0.755215464087799, P33=2.1388537303296355, P34=-1.195023558576381, P35=2.856482382928975, P36=0.6423079098544959, P37=2.5274447313541013, P38=2.0627817589040376, P39=4.168657164097081, P40=0.70709170029973, P41=0.6851428822213853, P42=0.694979469522465, P43=0.7043393022015574, P44=0.7508992589284433, P45=0.7523138614369659, P46=0.7566680166883468, P47=0.76187403930114, P48=0.8156952163523714, P49=0.8264544126040223, P50=0.7786344132541339, P51=0.7358642448917445, P52=8.13035226752236e-05, P53=1.6737220167805698e-05, P54=1.9372098585122264e-05, P55=8.851378030174102e-05, P56=0.0011019401788435995, P57=0.005620839411274812, P58=0.005534178638239824, P59=0.004545418620708452, P60=0.0050745485261282424, P61=0.00136212311653601, P62=0.0006263491656950837, P63=0.0008399907001032796, P64=0.0015987172134608143, P65=0.005141972321648415, P66=0.004442871034763905, P67=0.002564090692524641, P68=0.004142898510348768, P69=0.0045343514185750035, P70=0.001307604791356314, P71=0.0005037516386294093, P72=0.0008557601450812064, P73=0.0006867168902806597, P74=0.00027744701281818505, P75=0.0003071490533699721, P76=0.0007441379557733808, P77=0.0037532095531134274, P78=0.0033416542070122688, P79=0.0010597215307584593, P80=0.00026699457653512265, P81=0.0001758586563787553, P82=0.00012665676577356636, P83=0.00010166875767220581, P84=0.00015796560843343898, P85=0.0002518549175107232, P86=0.00024544305287912553, P87=0.00016476757543210804, P88=0.000149312990078649, P89=0.00012973430336650724, P90=0.00010396489464495585, P91=0.00013109181950571842, P92=0.00018481263387192228, P93=0.0001260500505964904, P94=0.00013462564327744545, P95=8.588781366377104e-05, P96=0.00010719543126461845, P97=0.00021083333948222446, P98=0.0003302820337892069, P99=6.028586160982048e-05, P100=0.0002062452862589107, P101=0.00032254509483228366, P102=0.0001770970747296329, P103=0.00010429921376117724, P104=7.301484267621649e-05, P105=5.9814328974075236e-05, P106=4.573405421123153e-05, P107=1.9389626678394602e-05, P108=5.451952806215205e-05, P109=7.905999965679105e-05, P110=3.658139172803233e-05, P111=2.7377360664905733e-05, P112=1.3724171511774485e-05, P113=1.5191556292562637e-05, P114=1.9477637170117425e-05, P115=3.9115248963587106e-05, P116=4.162724451798506e-05, P117=2.3997517016934084e-05, P118=2.8436058853123486e-05, P119=2.9737800239368497e-05, P120=2.3421040870590374e-05, P121=1.3632606877744043e-05, P122=1.807064588428991e-05, P123=8.21885836656514e-06, P124=8.83195730621887e-06, P125=6.636742676645605e-06, P126=2.3225581860649714e-06, P127=4.157193538286332e-06, P128=4.552970714564886e-06, P129=1.9791350154899195e-06, P130=3.911878715512484e-06, P131=7.733677959463319e-06, P132=7.962824364790741e-06, P133=1.3531331649345122e-05, P134=1.6352375087971362e-05, P135=1.2418264269887024e-05, P136=2.1142610376785145e-05, P137=1.4695914452439648e-05, P138=1.3950163861764747e-05, P139=1.0744978232928707e-05, P140=1.2552867017907684e-05, P141=2.5020255481004405e-05, P142=4.212523973203494e-05, P143=2.803300328743676e-05, P144=3.33006667011888e-05, P145=1.696694018178357e-05, P146=1.0931618553206877e-05, P147=7.2697486383746745e-06, P148=6.394018963689986e-06, P149=1.1809089777710231e-05, P150=2.0344644677165855e-05, P151=1.1896551670615845e-05, P152=1.9346359581970056e-05, P153=1.77897948133918e-05, P154=1.805604538805134e-05, P155=2.074254788436287e-05, P156=1.2045861921802593e-05, P157=1.3698319786643451e-05, P158=1.197380947253947e-05, P159=1.0179014952562478e-05, P160=1.2060286547485136e-05, P161=1.2604481989760424e-05, P162=9.041565760567513e-06, P163=6.218538716585546e-06, P164=8.707317620263842e-06, P165=7.135434440842471e-06, P166=6.142277836080515e-06, P167=5.313957609493387e-06, P168=4.625327517714297e-06, P169=6.974231276295911e-06, P170=5.496473375504364e-06, P171=4.60066287827206e-06, P172=4.340576461684817e-06, P173=4.4568989236176014e-06, P174=5.310146181892913e-06, P175=7.0591791606367315e-06, P176=1.1422967320485616e-05, P177=1.4877507341141205e-05, P178=7.106751701411375e-06, P179=5.262298549994503e-07),Row( P0=-567.0213535538744, P1=54.24803555461634, P2=-0.2833006007183958, P3=-3.2633305617805504, P4=-8.290632888247044, P5=-0.9287205115893926, P6=-5.232874406700888, P7=-4.90952158380948, P8=-3.2117523062543127, P9=-4.421957906708883, P10=-7.279297448539403, P11=-4.041578995257886, P12=-1.3467884284977751, P13=-7.084609527695207, P14=-5.358617305111633, P15=-2.55218562883507, P16=-3.882360876473149, P17=-3.2876495500408667, P18=0.3760936163143783, P19=-2.112279248501872, P20=0.8022437788847918, P21=3.1453655769320124, P22=2.714254256410251, P23=3.715110642900543, P24=3.0069268176551187, P25=8.17270571963174, P26=4.556986686152784, P27=6.584996608484184, P28=4.553795678337562, P29=3.5923925868062474, P30=0.8318875211961712, P31=1.7453757440642639, P32=-2.512733659398169, P33=2.6856089821481195, P34=1.8651496627867183, P35=4.332025437132906, P36=-0.3593211574623843, P37=-0.7237775016036321, P38=-2.4393904887866737, P39=2.4769728731745375, P40=0.597699330578967, P41=0.5552185619608649, P42=0.6157917625437005, P43=0.7213023304277097, P44=0.7796082862000437, P45=0.7188332177953745, P46=0.6832872466448219, P47=0.7072659483306092, P48=0.6653103769587347, P49=0.5925553297604801, P50=0.630908917479291, P51=0.6708958178237127, P52=0.0005234135729640603, P53=7.999061954004649e-05, P54=2.9581987437108273e-05, P55=3.2734138344367716e-05, P56=0.00022122180240689444, P57=0.0011355293522054468, P58=0.0022542267611652847, P59=0.009482711114886896, P60=0.06823584614068039, P61=0.18036469148094977, P62=0.18955891426323135, P63=0.19035592879468777, P64=0.10251029142661211, P65=0.01962327640022915, P66=0.005771630357163473, P67=0.0076753645514742765, P68=0.00826900301167606, P69=0.013768384410176346, P70=0.030129800852117927, P71=0.043910624459935106, P72=0.036219333232716186, P73=0.030356201165267455, P74=0.03582661440215403, P75=0.062369619563669924, P76=0.06211162068416094, P77=0.07703450077198705, P78=0.0421756997846724, P79=0.025109162030274388, P80=0.02947529513737133, P81=0.03809570116867217, P82=0.041136453910776805, P83=0.019751271032397797, P84=0.010514106904715393, P85=0.011799411845237626, P86=0.011030930632376002, P87=0.021482071892977902, P88=0.04551348833637387, P89=0.04768195837838774, P90=0.04407864654898769, P91=0.03420925454709747, P92=0.028563661335690937, P93=0.026075654522725242, P94=0.06169186906210583, P95=0.1207872855716605, P96=0.16466812050418322, P97=0.3312546458159304, P98=0.3043251014143117, P99=0.02274530948216256, P100=0.010945258086538413, P101=0.029703540383273917, P102=0.03784694906746823, P103=0.03820595366141317, P104=0.026896709957340025, P105=0.010534946226107816, P106=0.005425657224349046, P107=0.0036742949004267226, P108=0.01495759713969591, P109=0.018578265452839585, P110=0.013228646157666252, P111=0.005317384489708353, P112=0.0054890340987080975, P113=0.00402101792066099, P114=0.003035448118441792, P115=0.0033302811002660724, P116=0.0016493073807560524, P117=0.001113166600459136, P118=0.001815055545778101, P119=0.0011123700357252026, P120=0.0009924161843576655, P121=0.001121776819949143, P122=0.0009256793090762213, P123=0.0007898508078530831, P124=0.0004514466072864265, P125=0.0007301770243849873, P126=0.00033252397925991376, P127=0.000479645075549058, P128=0.0006984485897893416, P129=9.667752879153665e-05, P130=0.0001626982806700633, P131=0.00019092233032170637, P132=0.00022665422986157464, P133=0.0003609400789703981, P134=0.0005495136890465198, P135=0.0004311125554509234, P136=0.0006567165275445273, P137=0.0006436013014367953, P138=0.00046978777920973124, P139=0.0003118668153754475, P140=0.0002157084732069077, P141=0.00018398543631076655, P142=0.00015685883951378394, P143=0.00023402746138874092, P144=0.00023386468143388593, P145=0.0002703082827521488, P146=0.0003132185376593351, P147=0.0002417120769311779, P148=0.00014060778318227807, P149=0.0001456623198917227, P150=0.00029326630405202036, P151=0.00043682744059246706, P152=0.000538983647013944, P153=0.0007125819914655723, P154=0.0008102764872989709, P155=0.0008715983118127996, P156=0.0009582226870726777, P157=0.0006897945993501402, P158=0.0008211119833723275, P159=0.0009623153915681824, P160=0.0015445390169604137, P161=0.0017915852075764164, P162=0.0013164676094546165, P163=0.0005006728603450816, P164=0.0004773709643005776, P165=0.0003894736417401516, P166=0.00036240098799206254, P167=0.0005035750684455989, P168=0.000656660465868944, P169=0.000605026766105493, P170=0.0006919081892197974, P171=0.0007034981336777969, P172=0.0006332624511128292, P173=0.00042602885715063027, P174=0.0007600965829731853, P175=0.0008273290236457796, P176=0.0005843965512277547, P177=0.0004958579027769139, P178=0.0002179821141956982, P179=1.1092353798957416e-05),Row( P0=-508.89814371006236, P1=44.1008409237977, P2=-5.2910567410841916, P3=2.897256264423803, P4=-4.18549563393247, P5=-2.680732476604153, P6=0.240225865557022, P7=-0.10349099036542536, P8=1.3824330999920762, P9=3.6022527797809865, P10=-2.079084593069782, P11=-0.8487459551209395, P12=1.9768968698420433, P13=-0.3476096013471615, P14=1.6463247327995734, P15=5.546206451689656, P16=8.341779902981841, P17=10.143717779296175, P18=10.32140551795079, P19=12.128508645439206, P20=6.162035348871555, P21=6.835366139242312, P22=4.314596005733714, P23=5.9548590427705435, P24=4.694376336799492, P25=6.744577048130457, P26=-0.41726363816787476, P27=2.6961229329175995, P28=-0.6948859232727712, P29=3.17086069444741, P30=1.7761759519424765, P31=5.3255053639737095, P32=1.036276389165935, P33=3.3493174907225605, P34=0.6919166175521471, P35=1.597549878614217, P36=1.5328217277614027, P37=2.0730150844605926, P38=-1.7395460322494736, P39=0.96194921102431, P40=0.6944650865977235, P41=0.6513676293787175, P42=0.5969411306787507, P43=0.5821495683553891, P44=0.6533468885122766, P45=0.7069180140553328, P46=0.7434097993865642, P47=0.6958260989212296, P48=0.6378897435089507, P49=0.6545790829691679, P50=0.6834727406594139, P51=0.704323755623385, P52=1.874692223169124, P53=1.1453177669892478, P54=0.19566508898338386, P55=0.0059057691940320985, P56=0.0005439727329910083, P57=0.0010653004500692437, P58=0.0015959843563015587, P59=0.0064071951965642615, P60=0.01861488068172841, P61=0.036848343813271546, P62=0.22378633348033297, P63=0.3632842245585137, P64=0.1779042256209595, P65=0.4572478795837891, P66=0.9064542308225713, P67=0.8001492025933242, P68=0.29550594915544404, P69=0.04641294410753065, P70=0.01860496205656984, P71=0.012499575200316354, P72=0.015856219439135896, P73=0.02242115929873863, P74=0.06795606295277226, P75=0.12383258767367432, P76=0.09082554589581139, P77=0.1337162552645601, P78=0.3846269598158107, P79=0.47344892378973374, P80=0.29603913348131033, P81=0.4316016176399734, P82=0.7091067578913478, P83=0.20999069430114162, P84=0.1864698846674568, P85=0.23676615474643664, P86=0.08566772423515638, P87=0.06094888559408397, P88=0.03847039006425234, P89=0.019659653998480717, P90=0.16668705375512038, P91=0.6056047603631216, P92=0.5103872077060772, P93=0.2474563178535875, P94=0.1741921442464954, P95=0.08691754477465247, P96=0.10421696458637729, P97=0.4393451765055383, P98=0.27478050001203985, P99=0.017783498439012575, P100=0.09911591862814138, P101=0.17419014640466657, P102=0.05176879535795649, P103=0.01349380208809563, P104=0.02344322602674172, P105=0.20024815520300224, P106=1.0444982157597251, P107=0.3404458934121922, P108=0.05098915415631505, P109=0.03741647755765613, P110=0.010583403784428772, P111=0.021495648357852046, P112=0.030273070265402074, P113=0.11264127693715228, P114=0.07669615110510213, P115=0.015566831810358383, P116=0.01804809748400974, P117=0.01679723644583895, P118=0.025727356453471153, P119=0.01035711964191082, P120=0.004595860570318827, P121=0.01084733009089049, P122=0.008866130156009238, P123=0.0012812690004163308, P124=0.005065094457156037, P125=0.0040504019990887435, P126=0.0005687282112991949, P127=0.004047556934243275, P128=0.024436632661244708, P129=0.001618372466443486, P130=0.0023627583429508773, P131=0.00858109409763676, P132=0.003054848502090217, P133=0.0030915370627598994, P134=0.010665760088976253, P135=0.005646702850262168, P136=0.0053677434979184395, P137=0.00821176863878587, P138=0.0026228044109580517, P139=0.0023899448653894872, P140=0.0021988830365786147, P141=0.000908390524202688, P142=0.0008390464666382699, P143=0.0010256025335509248, P144=0.002017647010103409, P145=0.0025692222765819213, P146=0.001224426658688602, P147=0.0007761909374403936, P148=0.0004461204304532118, P149=0.000547564216030542, P150=0.0007022283328231789, P151=0.001227994471091642, P152=0.0011269705624072995, P153=0.0006148097864377268, P154=0.0012330125910017017, P155=0.0010540684922558914, P156=0.0010799223707933798, P157=0.0009412744898981812, P158=0.0006819878066206046, P159=0.00078667817343034, P160=0.0008761532629535185, P161=0.0006939390792459064, P162=0.0006606862881718955, P163=0.0008014375738585319, P164=0.0005407894407575517, P165=0.00032440118593650603, P166=0.0005277284802342641, P167=0.0004195183361636406, P168=0.00038751294610597824, P169=0.00030987386266440807, P170=0.00017758755013185117, P171=0.00020022456602300333, P172=0.0001852050023555347, P173=0.00032329945992437656, P174=0.0005247146137298957, P175=0.0005254720223978202, P176=0.0005140746705024477, P177=0.0003251697575402734, P178=0.0001026943616522391, P179=8.11988348821222e-06),Row( P0=-674.5058522839097, P1=61.519737981286966, P2=11.506715570436079, P3=5.111169157928095, P4=-1.4463010707837576, P5=-3.312025675039311, P6=-0.6788820771980504, P7=0.47762434811324317, P8=2.268315903145985, P9=1.567640040324087, P10=-0.7969957657855536, P11=-2.9952125865240693, P12=1.0851770750094651, P13=-3.1949409481077238, P14=-1.7863172433393515, P15=1.7790773907100852, P16=1.3505647275282484, P17=-2.7269717741356083, P18=-1.5895681126739507, P19=0.34242506297379904, P20=-0.8879250965836297, P21=3.307338715050726, P22=3.0355883618717794, P23=3.5978167366695337, P24=0.5827745624784951, P25=2.8212512579634628, P26=-3.200420858012371, P27=-0.3103372036771761, P28=-0.8550221853709631, P29=1.7861990222252504, P30=1.7368230207160618, P31=4.611708130763273, P32=0.4640950662909493, P33=1.7449585364465237, P34=-0.1824625211044597, P35=3.907130873001914, P36=1.4448618223831606, P37=1.3751894525807722, P38=-1.1474531013450666, P39=-1.147064433190761, P40=0.6886511789745748, P41=0.6682473166280966, P42=0.7027495875516039, P43=0.6570258674304792, P44=0.6461937361463824, P45=0.6863613555155691, P46=0.6969292590418326, P47=0.7073146542567118, P48=0.6316829880970837, P49=0.617203279222811, P50=0.6904683926916471, P51=0.7132772305613206, P52=4.351584044419332e-05, P53=3.128962196643346e-05, P54=8.14529982202867e-05, P55=7.243494149263526e-05, P56=0.0001981536799263799, P57=0.0013539400642656129, P58=0.007131401208623712, P59=0.014884329234941042, P60=0.020288104013426386, P61=0.027787010155261034, P62=0.06700703630373946, P63=0.0936192324255952, P64=0.024361350232730405, P65=0.013543875220971537, P66=0.028260110533129757, P67=0.026998160770716435, P68=0.025665307173110705, P69=0.022966224821374965, P70=0.012190719463590285, P71=0.009221336637418216, P72=0.01464402877630145, P73=0.013281136441215988, P74=0.030825202936104558, P75=0.06650205398891214, P76=0.037136525995644015, P77=0.013380900481554471, P78=0.017861519521900956, P79=0.016912017394146977, P80=0.013713273903719948, P81=0.01369036816254914, P82=0.01948727361278677, P83=0.01548697155214182, P84=0.016920088067480795, P85=0.022800343314682038, P86=0.026301747334620953, P87=0.03309149973423801, P88=0.012118344944808616, P89=0.0018343729704349578, P90=0.001068747180984717, P91=0.001298676740245761, P92=0.006237524482338826, P93=0.012381329763363933, P94=0.02030998159207274, P95=0.034847890606705595, P96=0.03545119082316492, P97=0.003517862477589771, P98=0.0010867617368856618, P99=0.0009548479470836999, P100=0.01074902648682607, P101=0.02696685873627076, P102=0.021715560611318612, P103=0.005163686315738972, P104=0.0005183982527591056, P105=0.0005925343470883992, P106=0.0005251711402216221, P107=0.001122759056644819, P108=0.010558343664117046, P109=0.006603026417488027, P110=0.0008681985801339032, P111=0.004255711344851463, P112=0.0033100834358945723, P113=0.004598813584222468, P114=0.0020601714507509613, P115=0.0007494643880943958, P116=0.0003855747206673078, P117=0.0004342289755805046, P118=0.0002294276950808555, P119=0.0005031671807793585, P120=0.00026907243999175443, P121=0.00032044322199445787, P122=0.00010102192788835812, P123=0.00022303524322137984, P124=0.0018608322610762888, P125=0.0004127159666208397, P126=0.0001069819673899058, P127=0.0002473298758549986, P128=6.664714880376986e-05, P129=7.409568761436244e-05, P130=0.00010048091104846428, P131=5.736538944249515e-05, P132=0.00012200722704102598, P133=0.00017680292775834592, P134=0.00033859815894462863, P135=0.0002035500348276369, P136=0.00018822016758138376, P137=0.0001588279442152623, P138=9.513742745122899e-05, P139=3.5548817671244406e-05, P140=3.5250173964908454e-05, P141=2.7222861400105123e-05, P142=2.6482279102908853e-05, P143=4.709953948246706e-05, P144=3.489473052170617e-05, P145=7.023061184274201e-05, P146=5.980319576875717e-05, P147=5.763855332103013e-05, P148=4.855642634328198e-05, P149=5.326501738129063e-05, P150=0.00010797639950665821, P151=0.00013666755112803504, P152=0.0002807625051830988, P153=0.00048525963896083025, P154=0.0006669598459460097, P155=0.0006427166640271741, P156=0.000551612530036816, P157=0.0004939490402161068, P158=0.0003546918129368617, P159=0.0004056404590063066, P160=0.0002831789723788458, P161=0.00020940646204826462, P162=0.0001697186381667472, P163=7.013738620178438e-05, P164=8.199763915932594e-05, P165=0.00012879434731432503, P166=0.00013683273360379068, P167=9.746519453806532e-05, P168=0.00013022119988650646, P169=0.00011832718625594625, P170=0.00010457688047665306, P171=8.776446199114503e-05, P172=8.701955550484927e-05, P173=0.00015530275225868827, P174=0.00021374801139083854, P175=0.00034224586238385587, P176=0.00029949334864617444, P177=0.0001789016500006264, P178=9.308482399338223e-05, P179=9.104050431586846e-06),Row( P0=-684.0839454273271, P1=63.23893938060573, P2=5.769457200568266, P3=6.2854781476996955, P4=-1.132684342321168, P5=-3.995368182227807, P6=-1.294341732368723, P7=3.1937659023774714, P8=4.647296397499959, P9=1.1759902672611413, P10=-0.22566942104075222, P11=-1.7104755764000679, P12=3.0984984710436763, P13=-0.43700753741254544, P14=-0.13726562745363302, P15=4.438415837829576, P16=1.0578522487875486, P17=1.073510650074054, P18=-0.5298583715794842, P19=1.5974155831725616, P20=3.4330190812389483, P21=3.647845621637714, P22=5.8983282487751465, P23=8.154895944452019, P24=5.7439560760338235, P25=7.834127678766633, P26=1.402047719034955, P27=1.343644620084087, P28=-1.0777386216069984, P29=0.5460054892567187, P30=0.9024697486848403, P31=3.9475564348573897, P32=0.5314155642465204, P33=1.9283865231564135, P34=-3.565736936604501, P35=0.001283089150047327, P36=-1.4708483344882748, P37=1.5964591072510015, P38=0.23744113586494167, P39=-0.5153566466134897, P40=0.6989543941812386, P41=0.6909981917955006, P42=0.6642096384815008, P43=0.7000480606094248, P44=0.7139997071313455, P45=0.8064199553604601, P46=0.7954480289137188, P47=0.7398894104667402, P48=0.6521098929789505, P49=0.6600765622596513, P50=0.6472460099643188, P51=0.6436878541017638, P52=9.307320386690538e-05, P53=2.883005373227409e-05, P54=3.901249937876169e-05, P55=1.8834793076228227e-05, P56=7.678794714662492e-05, P57=0.00039054453790899723, P58=0.0009065234435466046, P59=0.0009289188982432099, P60=0.002976221499786286, P61=0.01986129128840897, P62=0.07656235433989075, P63=0.05979397518183351, P64=0.010022876849117436, P65=0.007963098380037916, P66=0.01166601171283016, P67=0.007047578897964285, P68=0.002470009668226479, P69=0.0019181623510941288, P70=0.002220814379707963, P71=0.0029761966408934745, P72=0.012725880389197291, P73=0.02921890840615624, P74=0.020341969472244207, P75=0.013674651720965903, P76=0.017211344646153315, P77=0.015515326536410986, P78=0.004514941479521773, P79=0.008245076201956254, P80=0.024183189903341788, P81=0.02463167622702979, P82=0.015495872891593676, P83=0.03574259835136426, P84=0.04887063536942954, P85=0.03025004338300112, P86=0.012016408599753376, P87=0.01125488608571502, P88=0.0237418718214009, P89=0.01370550321285418, P90=0.002682319263841906, P91=0.005210387433501624, P92=0.05726030382160041, P93=0.04911519622765126, P94=0.011581189069868799, P95=0.0022018821272773883, P96=0.0003557373672625947, P97=0.0006265335070686006, P98=0.0005891256192144398, P99=0.0018925026226419284, P100=0.013786365358941163, P101=0.006384794822850212, P102=0.000590789416044359, P103=0.00030753119810447157, P104=0.0005372649840148534, P105=0.0015299877162132602, P106=0.0021742308807279967, P107=0.0005580790829512165, P108=0.0001589798906140312, P109=0.00045378577141168774, P110=0.0075266118270964, P111=0.00828981801400583, P112=0.00031959756829719513, P113=0.0005420884138132395, P114=0.003714671798762693, P115=0.0027555396753937045, P116=0.00027251128320009664, P117=0.0008906579424921309, P118=0.0007439240764117699, P119=0.0001811494240291686, P120=0.0002144544696303912, P121=0.0002769123783855324, P122=5.7228156138567755e-05, P123=5.157132164043631e-05, P124=5.8797488667893456e-05, P125=2.59626009925706e-05, P126=2.3477301934810113e-05, P127=2.512267684653983e-05, P128=4.538947526812852e-05, P129=2.632127819016848e-05, P130=2.261658021940091e-05, P131=8.459101873553194e-05, P132=2.293285527130917e-05, P133=8.547833999262947e-05, P134=4.62677536911319e-05, P135=3.4722407860223416e-05, P136=3.5574721541139e-05, P137=3.9084418081269664e-05, P138=2.8577193077877665e-05, P139=2.8383631025141478e-05, P140=2.841140082234007e-05, P141=2.9795745647844866e-05, P142=2.3251495983535636e-05, P143=2.238904990062061e-05, P144=2.0475308059561513e-05, P145=2.4130578010129097e-05, P146=2.6023488195528445e-05, P147=8.23058767161568e-06, P148=8.834284740817305e-06, P149=8.29392855878346e-06, P150=1.0759328250256635e-05, P151=2.452076129601946e-05, P152=1.717006969629783e-05, P153=2.0833106164898028e-05, P154=2.0420498997662144e-05, P155=2.1928195916183817e-05, P156=2.2233447078403048e-05, P157=1.4371186289592568e-05, P158=9.622908129583533e-06, P159=1.2949492518196968e-05, P160=1.310768956426119e-05, P161=9.426035427493683e-06, P162=5.21266920938473e-06, P163=6.936576707867918e-06, P164=6.705858040049934e-06, P165=4.336382086039108e-06, P166=6.408453565051761e-06, P167=1.1535208409911127e-05, P168=4.385237732552677e-06, P169=4.359392819175069e-06, P170=3.4891024365531724e-06, P171=2.4262123514172113e-06, P172=2.3952577034915427e-06, P173=3.4108131176217904e-06, P174=4.039151874623805e-06, P175=7.607134588855299e-06, P176=7.971043476407674e-06, P177=6.968444129316686e-06, P178=3.101177558736958e-06, P179=2.2247040453578652e-07),Row( P0=-760.9281187502227, P1=65.62575578644885, P2=20.405931316820016, P3=21.05584321691774, P4=6.050812252258026, P5=4.297022208243649, P6=2.184483021334719, P7=4.318924935679762, P8=1.3637015872757379, P9=2.975202986480382, P10=-2.2117385019453297, P11=-1.0663334172947154, P12=2.4137904842877633, P13=-3.230883281257255, P14=-1.0939020028438773, P15=3.1422812799944175, P16=0.18720231642372603, P17=-1.0744947253071782, P18=-0.5488080547064906, P19=1.2880312805445813, P20=-1.0442393327485984, P21=-0.1559110652910299, P22=-3.218258494018864, P23=-1.1788710735401975, P24=-4.1317240685589045, P25=2.272230246861336, P26=-2.493219259150627, P27=0.06850037007620778, P28=1.3288817499939376, P29=2.815490889131771, P30=2.1576003461854305, P31=6.229455229082538, P32=1.2727074113880816, P33=5.631997755675382, P34=2.8308252128358466, P35=4.9248542368203205, P36=3.0766672867825453, P37=5.071650801522114, P38=2.5147845655148173, P39=4.393607308010403, P40=0.7216230951793653, P41=0.6791202710798488, P42=0.6647530586345711, P43=0.6729859589947487, P44=0.7069075932246065, P45=0.680982813910395, P46=0.6798958843502793, P47=0.6950045300491733, P48=0.7699122395989026, P49=0.799058222739697, P50=0.7947021786884894, P51=0.7562017596958678, P52=7.142482684274999e-05, P53=1.076767047305753e-05, P54=9.297588975814376e-06, P55=4.958149358911231e-06, P56=6.14133465695948e-05, P57=0.00114353248900316, P58=0.0036243201914678163, P59=0.006408695758298478, P60=0.005649715300855381, P61=0.0011784275819159834, P62=0.00017866659080157055, P63=0.00016992405376849224, P64=0.0008583596984706045, P65=0.0028980786339599376, P66=0.003532672186041923, P67=0.004029046655455763, P68=0.0035496900132178143, P69=0.0031787315539937313, P70=0.0011612458005589253, P71=0.00021393593208370867, P72=0.00027854642076012744, P73=0.0002714386467406162, P74=0.00046493164825659594, P75=0.0008399662263658668, P76=0.001236200050275972, P77=0.0016082184049566977, P78=0.003372967699847796, P79=0.001901849928493405, P80=0.0002812233404012192, P81=0.000131979961368674, P82=0.00017169929807776477, P83=0.00010000302599659231, P84=0.00019555567788971792, P85=0.0002443797928512114, P86=6.870765552088178e-05, P87=5.1552147526707766e-05, P88=3.762085112113698e-05, P89=2.5813475411533957e-05, P90=3.6727985185316284e-05, P91=9.352127125766756e-05, P92=7.051380357059352e-05, P93=3.025303679939804e-05, P94=2.6677190623489177e-05, P95=2.3377293359096445e-05, P96=1.8025974444958054e-05, P97=5.7350932752099223e-05, P98=3.579624253206081e-05, P99=1.709338125534314e-05, P100=1.0203944593213298e-05, P101=4.062359747433429e-05, P102=0.00010885606835064238, P103=2.6508634080928e-05, P104=1.2363807747095849e-05, P105=1.7588889491868766e-05, P106=4.236256944306828e-05, P107=1.3381333053730206e-05, P108=1.0858292358711621e-05, P109=1.6448106272532453e-05, P110=2.1994051085328228e-05, P111=1.2393645939381987e-05, P112=1.2096028116123227e-05, P113=1.865954354918795e-05, P114=1.7862733408370262e-05, P115=1.006972432457924e-05, P116=1.338049507263426e-05, P117=7.435512259597867e-06, P118=6.259802743413931e-06, P119=8.12024391658903e-06, P120=6.964278616478787e-06, P121=7.448807699636323e-06, P122=9.740550785527899e-06, P123=5.206998398997212e-06, P124=4.732019370748216e-06, P125=4.9663280914542745e-06, P126=1.783321994255811e-06, P127=9.444487775406068e-06, P128=9.141181433047366e-06, P129=2.719062973603875e-06, P130=3.9767464231712285e-06, P131=4.525741605035559e-06, P132=4.741153272621728e-06, P133=1.4139153075656415e-05, P134=1.5171516695017673e-05, P135=1.5766042791210467e-05, P136=1.4397668012131921e-05, P137=8.640488750330116e-06, P138=8.630479129841105e-06, P139=5.866907492776135e-06, P140=1.706938607005417e-05, P141=3.230894532276355e-05, P142=3.212047377688418e-05, P143=3.3999806060556956e-05, P144=1.2839609034689433e-05, P145=9.695336207119603e-06, P146=5.44705040953007e-06, P147=5.923865890546539e-06, P148=5.2058186041308305e-06, P149=4.020270763605242e-06, P150=5.261416711401931e-06, P151=8.118635731794673e-06, P152=1.1655558842886019e-05, P153=1.578905094570261e-05, P154=1.0920204638880718e-05, P155=1.0303652480966622e-05, P156=1.826616339826145e-05, P157=2.4229430289914164e-05, P158=1.4467496973317578e-05, P159=1.231340576973017e-05, P160=8.291519978567723e-06, P161=6.6289666487339216e-06, P162=7.612325869369737e-06, P163=5.5958508472213875e-06, P164=4.8156105737615345e-06, P165=5.1625156904920415e-06, P166=5.234339955205597e-06, P167=6.820584662869115e-06, P168=3.324497919709737e-06, P169=1.2116977522192658e-06, P170=1.4263511838712465e-06, P171=1.0525465409037796e-06, P172=1.8453432250363334e-06, P173=2.1539791684861853e-06, P174=2.8487456148604826e-06, P175=2.996704053734761e-06, P176=7.012333418635423e-06, P177=7.769134229917303e-06, P178=2.010470458372838e-06, P179=1.002451403684386e-07),Row( P0=-626.8708752528287, P1=58.28171497845551, P2=2.955042446140679, P3=2.7308138596562053, P4=-6.003092013941127, P5=-7.0654872520106915, P6=-7.462494036046176, P7=-1.2621261271813438, P8=4.208113963828237, P9=-2.1719478478922563, P10=-5.038144848874756, P11=-2.3864691638958657, P12=-0.6035460979301924, P13=-3.042045263672659, P14=-1.3600731859700959, P15=0.460148521787463, P16=0.13448593814705745, P17=-1.9889960461846576, P18=-3.2797733337778605, P19=3.411350130966016, P20=-0.4776228090325109, P21=3.8205520122523824, P22=4.628396233669454, P23=5.529651526635557, P24=2.9309236240918812, P25=6.710168972756187, P26=1.7594226777634878, P27=2.5790017648045565, P28=2.717488463323618, P29=3.587034429895999, P30=-0.4801804000944465, P31=2.3976543967663804, P32=-2.330002287084219, P33=0.9556919304757744, P34=-1.7231608087553951, P35=-0.48668990445292043, P36=-1.2260405520490634, P37=-1.095796326520585, P38=-1.2999679108770394, P39=0.8337876462430436, P40=0.6606000967005181, P41=0.6564889389513989, P42=0.6802669332129246, P43=0.7293567111617516, P44=0.7554716072547409, P45=0.7897401324654532, P46=0.7669721187154053, P47=0.7125040639145134, P48=0.6755482096028471, P49=0.6520640350266814, P50=0.6283316084934165, P51=0.6498748833918621, P52=4.9290791658751565e-05, P53=4.163548293691993e-05, P54=5.933878881592207e-05, P55=2.8733667490613425e-05, P56=0.00016120135278032752, P57=0.0007454787071628587, P58=0.0015086343079745114, P59=0.004759537729330284, P60=0.02785937666870805, P61=0.09947962641323307, P62=0.2388690441066363, P63=0.25754123743320845, P64=0.0847183284270084, P65=0.019083009794586197, P66=0.005465050619836536, P67=0.005505888761663571, P68=0.006556182907187712, P69=0.012044593332442002, P70=0.02062596808816971, P71=0.025658111091065722, P72=0.04624856402092015, P73=0.06525814654154827, P74=0.05603149279534974, P75=0.04555956490987192, P76=0.03944978875120024, P77=0.05053473980381947, P78=0.027999721561333304, P79=0.01861459714074612, P80=0.033397528592686576, P81=0.05480880188250462, P82=0.05270882719196498, P83=0.04923357400671681, P84=0.08015880681253613, P85=0.0887303484038188, P86=0.017096252484686377, P87=0.030439659381897285, P88=0.06652049197129421, P89=0.11288884318772395, P90=0.04981006146273018, P91=0.029487181684385744, P92=0.07020334147697763, P93=0.009161719944252385, P94=0.001897407864073566, P95=0.003913093377354139, P96=0.01136339209693835, P97=0.030937440904960506, P98=0.08119237046767948, P99=0.019923747108518523, P100=0.0013556615182971422, P101=0.0006815889702937688, P102=0.001985320687965627, P103=0.00285254151854543, P104=0.006270256896671435, P105=0.006233736906524581, P106=0.0029531684809343, P107=0.0004639110671127867, P108=0.002793323359201925, P109=0.00911221518918736, P110=0.01763635265195583, P111=0.02056937566698343, P112=0.002068981902081153, P113=0.004336184795595842, P114=0.0020086785957246986, P115=0.0014854188495898814, P116=0.0016121723792735028, P117=0.003289476950301156, P118=0.0009742953973986907, P119=0.0009540200609100449, P120=0.0012949537280328846, P121=0.0007113046972898439, P122=0.0006014852995320358, P123=0.00031665024183335496, P124=0.0012138583633880958, P125=0.000934469492570717, P126=0.00010790693782236312, P127=0.00016856653468972865, P128=0.00020433154012135002, P129=0.00011375658396234998, P130=0.0001577940737402876, P131=0.0002660128305881611, P132=0.00033627624174493726, P133=0.0004411541094220392, P134=0.0005956248348557733, P135=0.00039894597345430837, P136=0.0003743960419781886, P137=0.0001508644216621136, P138=0.00010385792173451967, P139=6.16384147801656e-05, P140=5.979588971822471e-05, P141=0.00012847040874018828, P142=0.00022855744238481606, P143=0.0002479377253803616, P144=0.00020207589129323615, P145=0.0002117372708085127, P146=0.00016852054965268312, P147=0.00012729499643684988, P148=0.0001012668589045242, P149=0.00011906523158238243, P150=0.0001610465304298043, P151=0.00020216270108305113, P152=0.0005823268823893222, P153=0.0003832078116376729, P154=0.0005123337312599736, P155=0.0004447823538145508, P156=0.0004220766352732978, P157=0.00033132961693851405, P158=0.0003213476538861855, P159=0.00043664627431638445, P160=0.0004470219759092888, P161=0.0003568571895020017, P162=0.00029622880851809687, P163=0.00025047512109304196, P164=0.0002213346393087153, P165=0.00017339678988538505, P166=0.00016220397227939757, P167=0.0002609027621215375, P168=0.00021174150517943766, P169=0.00015846193779738642, P170=0.0001407256346092001, P171=0.00014626991964459307, P172=0.00012802580681025793, P173=0.00015028298253330847, P174=0.00027765509331750235, P175=0.000462111364911932, P176=0.000667968591157155, P177=0.0005960343349996543, P178=0.00023911163142313, P179=1.7554029246994168e-05),Row( P0=-588.3037914695831, P1=60.13257081661113, P2=6.500225495689388, P3=8.115950696025275, P4=-4.742506410825123, P5=-0.8445246197029852, P6=-0.48513279844448154, P7=-1.0993965588761758, P8=2.69148681204786, P9=3.1492440958761017, P10=-3.1219443119173027, P11=2.4119925361362498, P12=3.473050437954612, P13=-4.549057698542116, P14=-0.24725389796792546, P15=1.2180754030145238, P16=0.8702632225430882, P17=0.3038330306844454, P18=2.0953429288324283, P19=2.0507116880296397, P20=-2.0957560119254226, P21=1.6769629813345952, P22=-1.483693400832581, P23=0.49750298105044016, P24=1.4165804475052743, P25=5.670523629079634, P26=5.32563642150911, P27=8.922969994165378, P28=5.489006502754204, P29=6.430341472037903, P30=4.044801698138987, P31=3.573456389601383, P32=-0.8582202084487418, P33=3.5431588698928014, P34=0.027567134931908783, P35=4.15188372584134, P36=0.44570319697962746, P37=1.3929793442296525, P38=-0.46285963003990804, P39=-0.4304834827496729, P40=0.6152732702784941, P41=0.630827634877108, P42=0.7308595806333947, P43=0.7385223509344431, P44=0.7035717787617028, P45=0.6774487523830689, P46=0.6949758935279651, P47=0.6663355842136881, P48=0.6847371248471289, P49=0.7067540302065987, P50=0.6960371044966019, P51=0.6816482814842887, P52=0.2485343456422867, P53=0.03514816319223906, P54=0.002671509156496298, P55=0.0001983623868640334, P56=0.00015510421004532893, P57=0.0005007757281791697, P58=0.0014098207960183571, P59=0.007384160497504496, P60=0.05753835785880316, P61=0.12051659439407764, P62=0.06894202714294152, P63=0.02048035254845212, P64=0.029637483046221033, P65=0.06685093618775681, P66=0.020748405860484584, P67=0.005383783375186907, P68=0.008570119429409934, P69=0.03330130733695183, P70=0.07276540829386077, P71=0.050157296059748065, P72=0.016073410426336553, P73=0.007589842681389342, P74=0.0068271397209693045, P75=0.007911769542472984, P76=0.010274842150508733, P77=0.038451839696437666, P78=0.08884602390547397, P79=0.13214895502538015, P80=0.06988390195413016, P81=0.04621990074694516, P82=0.049627287458580735, P83=0.013280069493353199, P84=0.003547105551459255, P85=0.003990755884910597, P86=0.002983803941442961, P87=0.0065716382746506, P88=0.01751413986020237, P89=0.010390318961066191, P90=0.006394963424528396, P91=0.004434456182563137, P92=0.008013512552570903, P93=0.011570117934499594, P94=0.026011613700838557, P95=0.01317476177625249, P96=0.005270563635046262, P97=0.004300013790517745, P98=0.003811374847418167, P99=0.0030637665629906245, P100=0.005220662064158531, P101=0.0069605751748324654, P102=0.003339176597566509, P103=0.0018004292019152626, P104=0.0022615730722264716, P105=0.002323192834804911, P106=0.0057930780759670175, P107=0.002954312660755909, P108=0.0011075214657770404, P109=0.0011123573473542756, P110=0.0012735643465574694, P111=0.0029281929760582993, P112=0.0014346385767955898, P113=0.0008365426433151535, P114=0.0011267763177689107, P115=0.000858017780773834, P116=0.0009243166682532262, P117=0.0019438765160104182, P118=0.0006515094697282422, P119=0.0005002254126253166, P120=0.0007500796806694491, P121=0.0003713484978234884, P122=0.0005848348479485092, P123=0.0001995091524039279, P124=0.00015754007816736555, P125=0.00014595387468129174, P126=7.137490890597754e-05, P127=0.00019513108313450397, P128=0.00022762885690751102, P129=8.662452919430894e-05, P130=0.00011907374001959689, P131=0.00018120829591050704, P132=0.0001279252225860286, P133=0.0002765179543390383, P134=0.0003583538184437367, P135=0.0003506979852735589, P136=0.00040449564991910513, P137=0.0005144171172382641, P138=0.00027101916797537965, P139=0.0003003528779926814, P140=0.00022816730003706508, P141=0.00018043218799098412, P142=0.00013347365864422315, P143=0.00017203927593877324, P144=0.0002794377858404233, P145=0.00020081051960531488, P146=0.0002634716305324852, P147=0.00013930832873985764, P148=6.608546583648496e-05, P149=4.364792207325422e-05, P150=3.835127890571506e-05, P151=6.835232379481581e-05, P152=6.0721563739936484e-05, P153=0.00010567543283004918, P154=0.00012790400082682675, P155=9.539865865021232e-05, P156=9.359535492373399e-05, P157=0.00025127650656809117, P158=0.00019953723546202996, P159=0.00037116381475174907, P160=0.0003752846443557043, P161=0.00044494537368953334, P162=0.00033501458750565823, P163=0.00013800937040581873, P164=0.00016171828389413892, P165=0.00014054130345713425, P166=0.00011122144764479889, P167=9.716814693353289e-05, P168=0.00017265807155146496, P169=0.00018172682759112638, P170=0.00011839435270422727, P171=0.0001143991157261429, P172=8.531461538005041e-05, P173=4.3750294585303965e-05, P174=3.0379572732393777e-05, P175=4.858758680156741e-05, P176=6.868774018797187e-05, P177=6.471207912870124e-05, P178=2.567162218416103e-05, P179=3.143072750007629e-06),Row( P0=-544.8308384336791, P1=43.80859348973309, P2=-5.43358948374235, P3=-4.9341999965490615, P4=-8.498934618276309, P5=-6.630062586531858, P6=-7.383265460567479, P7=-3.3507077954840856, P8=-3.3382311835360063, P9=-1.2177199729237989, P10=-3.8374969073527425, P11=-5.988943510280667, P12=-0.7975303754760515, P13=-3.908529876717905, P14=-4.424504640037751, P15=-2.271817305796269, P16=0.5214085109339573, P17=1.2178203666985739, P18=3.7542190832383486, P19=9.604048787398174, P20=4.473507113515584, P21=9.96819797986924, P22=6.360442233383055, P23=3.664849636599286, P24=1.0651975506722124, P25=4.243154623801947, P26=0.1400223814887326, P27=2.4941724565671843, P28=-0.4396257999342674, P29=0.8492529491645636, P30=-1.2007796654404774, P31=2.6740394188687557, P32=0.27317956454238607, P33=1.3283259641956946, P34=0.30601226883413385, P35=0.7074565213655414, P36=-0.08384547150639668, P37=1.0275785958634573, P38=-1.8842504128585806, P39=0.05151523576822453, P40=0.6746086622973242, P41=0.623444152800291, P42=0.64276742210202, P43=0.6470370733583447, P44=0.6596712682815605, P45=0.6621378871686591, P46=0.6704132803102397, P47=0.6222502780705602, P48=0.6242863812236894, P49=0.6319615449111007, P50=0.6283676148841302, P51=0.6595661970796203, P52=0.0012811806054417595, P53=0.0007380972673458894, P54=0.0002947620695127692, P55=0.00012030715626750635, P56=0.0001359940224489765, P57=0.00023074114684407544, P58=0.0016603765125216456, P59=0.006112755961063232, P60=0.021525372697978128, P61=0.0855762699669968, P62=0.27207519506377226, P63=0.3975933802365567, P64=0.33295664298687955, P65=0.3018237826291154, P66=0.1716984073710915, P67=0.06391110633239466, P68=0.017086697183767634, P69=0.014800491820293435, P70=0.01583288354737631, P71=0.018939202354166927, P72=0.04487905442016991, P73=0.05490539249955475, P74=0.1295097750451465, P75=0.15847699545851707, P76=0.2096431593409515, P77=0.549492059854177, P78=0.6255740557455932, P79=0.3996791373505114, P80=0.1638683475050859, P81=0.1264822744190309, P82=0.15280498072238605, P83=0.1487101864822724, P84=0.1868481023505376, P85=0.2082760582132883, P86=0.10411066319941176, P87=0.08527111993559315, P88=0.1439004354541062, P89=0.3001221246110496, P90=0.25024020048760737, P91=0.2190017991689598, P92=0.42437576010394806, P93=0.22156977432298647, P94=0.3118098517059332, P95=0.2812783860083389, P96=0.09876934811309515, P97=0.05286377892058528, P98=0.22282829081320632, P99=0.2615031777856649, P100=0.8304473907033476, P101=0.8353181224776003, P102=0.1374134285672589, P103=0.019416673419136856, P104=0.004261797781048904, P105=0.04775370173867379, P106=0.06129357071158696, P107=0.044869940878220375, P108=0.14150901589725726, P109=0.14571226833308706, P110=0.16587594168742076, P111=0.19362851560017222, P112=0.05227831680292273, P113=0.048554298520673785, P114=0.033719628844689965, P115=0.017351781167008595, P116=0.019820648613137933, P117=0.01987773722573097, P118=0.0032480146644780637, P119=0.0082977830010675, P120=0.007397154743531939, P121=0.001625883743553809, P122=0.0016430490323558838, P123=0.00329391688655207, P124=0.00521519842920067, P125=0.001441174561861588, P126=0.0016970452352923705, P127=0.002281256649136422, P128=0.0008235763956921467, P129=0.0010539608735343577, P130=0.0009742384152950347, P131=0.0014958913746637979, P132=0.002047258164910148, P133=0.0031270786811444565, P134=0.003490205149604182, P135=0.0026498158573839033, P136=0.0013564204518347553, P137=0.0016010340369559427, P138=0.0011208356745804316, P139=0.0005882423386205866, P140=0.0001886092530996262, P141=0.0001628554254651957, P142=0.0002092461343815373, P143=0.0004613593144149257, P144=0.0003112743019277893, P145=0.0004670817266759894, P146=0.00038736975819265946, P147=0.00037266611797533324, P148=0.0002475774793054382, P149=0.0002477165457126976, P150=0.00020918368180888476, P151=0.00021900209929647747, P152=0.00029328441477667237, P153=0.000351293339216708, P154=0.0005577737871837583, P155=0.0020301068481183644, P156=0.0020506335746159994, P157=0.0028368485623778005, P158=0.001830934237857393, P159=0.0017609110624323319, P160=0.0016599187135436673, P161=0.001274225948230555, P162=0.0014661872357745044, P163=0.0014627185316488176, P164=0.0009970664165165103, P165=0.0006326323842390023, P166=0.0005957886470375472, P167=0.0005059374422781435, P168=0.0005089033860790644, P169=0.0007088995882302409, P170=0.0006551010120342617, P171=0.0006341652737866139, P172=0.0006746890676035222, P173=0.0007703917398674967, P174=0.0008094254953987776, P175=0.001352249697945535, P176=0.00132618174102758, P177=0.0008704312773928189, P178=0.0003659867488256138, P179=3.23014599292116e-05),Row( P0=-732.6176270768026, P1=90.04264862994744, P2=23.67602520752613, P3=20.812037039289827, P4=6.120821602371447, P5=1.7905144188155877, P6=0.8573636298045533, P7=4.6400476149292, P8=4.712499666393673, P9=2.9095531512523496, P10=-1.596358990745739, P11=1.5977396981439798, P12=0.5239605288681967, P13=-3.830849216487467, P14=-3.1416718772640473, P15=1.7901013129689478, P16=-0.22741167871227055, P17=-2.377608501616408, P18=-1.6727683968469595, P19=-0.25436781100575545, P20=-2.7751330592831445, P21=-2.745800326447715, P22=-3.806309832260036, P23=-1.862070680395315, P24=-4.177484858164391, P25=0.24332671575984255, P26=-5.700909175066173, P27=-1.8538667215493203, P28=-2.003315728942723, P29=-0.22804817767482077, P30=-0.308411561968357, P31=2.676423253969639, P32=-0.9013451989824903, P33=2.3065652333660815, P34=-2.1730222709355047, P35=0.4259637925509042, P36=0.8495493952133746, P37=3.0047205178175678, P38=1.7095747602196523, P39=5.297256672758108, P40=0.6598558955480404, P41=0.6072909947156049, P42=0.615612805256369, P43=0.6446903114740667, P44=0.6802039397396448, P45=0.7341581417922334, P46=0.7829618564557729, P47=0.7720383338763115, P48=0.8096758075270084, P49=0.7641711452943117, P50=0.690151559519346, P51=0.6582931829025888, P52=9.420551295905788e-05, P53=1.5670296790109013e-05, P54=1.9531747870769477e-05, P55=0.0001053496405693492, P56=0.002292143287716123, P57=0.01305807750932677, P58=0.009695940049241618, P59=0.006037779224953613, P60=0.0035701954288220547, P61=0.0009723720404809912, P62=0.0019016478211535122, P63=0.0029930420973504493, P64=0.00530114942018467, P65=0.006156158980451854, P66=0.0038863137886187044, P67=0.005625098102285777, P68=0.00454229264045691, P69=0.0022984041504642777, P70=0.0013354267331534921, P71=0.0011323060261501845, P72=0.0005998834916391405, P73=0.00037426878355725605, P74=0.0005446316912942866, P75=0.001212864203394623, P76=0.0012597884244604468, P77=0.0018428788277010917, P78=0.0014757150158033337, P79=0.0010199372755521653, P80=0.0005550775145909124, P81=0.0003341681101526021, P82=0.0004296070848724393, P83=0.0008228637695466682, P84=0.001087684853298707, P85=0.0007956190814807939, P86=0.0002690764002569216, P87=8.611727346224805e-05, P88=0.00012244839908666632, P89=0.00018167951684034683, P90=0.0002317427241299517, P91=0.00022620453647137397, P92=0.00027207050850889966, P93=0.00024296345519499326, P94=0.0002160771783249562, P95=0.00028856140344303094, P96=0.00025097505614479, P97=0.00019039010047118847, P98=0.000454829855607386, P99=0.0003263958302346348, P100=0.00022832976452452015, P101=0.0003111089837097034, P102=0.00032128776406574916, P103=0.0002084141274521829, P104=5.986737508929648e-05, P105=8.177037763794369e-05, P106=9.480903423502242e-05, P107=5.582929233907512e-05, P108=0.0002721815388212845, P109=0.0002219564505914245, P110=8.65023054658777e-05, P111=9.005436201917625e-05, P112=2.038040949162264e-05, P113=3.4410424515842175e-05, P114=4.958889166306637e-05, P115=4.019558481800573e-05, P116=6.379529218434849e-05, P117=3.1816480085426994e-05, P118=4.28171777586952e-05, P119=4.027033224925929e-05, P120=4.08601986574725e-05, P121=2.23041861749823e-05, P122=2.4325716604116546e-05, P123=1.2686795571132566e-05, P124=1.3996406766167866e-05, P125=1.2145711509141704e-05, P126=4.562006632332398e-06, P127=1.0754872169461199e-05, P128=1.4707405962334529e-05, P129=4.631926077209066e-06, P130=6.383609333512927e-06, P131=1.248477093076825e-05, P132=1.0826814356964922e-05, P133=1.4412913312520776e-05, P134=2.0642745463851757e-05, P135=1.8902998948853097e-05, P136=2.4328995876284756e-05, P137=1.694609289934143e-05, P138=1.7099570096651798e-05, P139=9.821168588860096e-06, P140=1.5115221539453449e-05, P141=3.329741244543175e-05, P142=4.4979097013404245e-05, P143=4.5735940202412536e-05, P144=3.804931327618203e-05, P145=2.65837487403545e-05, P146=1.368963202889077e-05, P147=5.950818983725317e-06, P148=3.447658751361899e-06, P149=6.392935817926162e-06, P150=1.4312542860688362e-05, P151=1.6626338254387553e-05, P152=9.41425289850739e-06, P153=1.6614560702851767e-05, P154=2.1603668805233952e-05, P155=1.3499241654202598e-05, P156=1.4366858434098302e-05, P157=2.72102730351748e-05, P158=1.706012654892505e-05, P159=1.2287348428039919e-05, P160=1.1522451295221204e-05, P161=1.4573012137995283e-05, P162=7.0848613721981496e-06, P163=7.0753155522986404e-06, P164=6.783463368042398e-06, P165=5.547186384559709e-06, P166=7.94323088594e-06, P167=7.503440803758374e-06, P168=6.219075202143462e-06, P169=3.444210367952703e-06, P170=1.7897381138239308e-06, P171=1.9927695993432666e-06, P172=2.4551517696127094e-06, P173=4.884930404944061e-06, P174=3.5568902747935825e-06, P175=3.302271371584749e-06, P176=4.336476071984427e-06, P177=6.2228841460044605e-06, P178=3.396414219278188e-06, P179=2.9191493559490616e-07),Row( P0=-510.4904223324354, P1=39.046549038020444, P2=-13.842342939130925, P3=-0.5586155897887772, P4=-10.580513109810003, P5=-8.180264469753233, P6=-5.638715201827529, P7=-4.485739296054917, P8=-0.522479396381516, P9=-0.2115312065117567, P10=-8.885557544069915, P11=-5.0620738852720315, P12=0.0029900845049991753, P13=-1.447383253574939, P14=-1.3134538592799438, P15=4.079210226657698, P16=7.921139531804994, P17=11.346298669596232, P18=9.902640278723815, P19=12.850228871311321, P20=4.520302561583742, P21=2.491040572929748, P22=-0.5505752661072958, P23=2.1992416067268987, P24=1.503357949611901, P25=7.245574957973195, P26=2.995091608550004, P27=-0.9933609688157999, P28=-2.48165012260605, P29=0.5587102588995702, P30=1.7545592604026914, P31=6.058971214137434, P32=-0.2682918510229614, P33=1.8253956646638119, P34=-0.502249271318632, P35=4.595708879924172, P36=4.822723379230529, P37=0.6980959547721948, P38=-2.421030495727424, P39=-1.7038974140104997, P40=0.6019516272761787, P41=0.5470872708788362, P42=0.6065916714703554, P43=0.6305155310910363, P44=0.668827175135598, P45=0.7748311956169831, P46=0.7406955284236212, P47=0.5826106167083188, P48=0.5570211654000339, P49=0.6268509437978552, P50=0.7255447224458375, P51=0.678998400429327, P52=0.11655388709127334, P53=0.01850540468665496, P54=0.0009117046878822888, P55=0.0002135211187271408, P56=0.0002236175788808968, P57=0.0003082944087021576, P58=0.0005449929997017868, P59=0.0013452674250240629, P60=0.0041078563537931545, P61=0.013864498582583939, P62=0.06565785473423298, P63=0.133671044363923, P64=0.15447147647381154, P65=0.9676907153392587, P66=2.3374544171816987, P67=1.8627673876265967, P68=0.5682390699406519, P69=0.2644255525701968, P70=0.14061070012880375, P71=0.03278041740447475, P72=0.00753549788073486, P73=0.006725080889071249, P74=0.008303151742190333, P75=0.012483355919519656, P76=0.015790800700281802, P77=0.03989580713720577, P78=0.10966356723421411, P79=0.39360384745593785, P80=0.7686074951578356, P81=1.032719154523971, P82=1.745241496695924, P83=1.4283438516150788, P84=0.536037283072565, P85=0.24131756952282804, P86=0.03514500478603773, P87=0.021431348596178706, P88=0.03429204902750982, P89=0.043167442818517095, P90=0.081174674243023, P91=0.238382212335822, P92=0.6757758446547013, P93=1.7124953808312144, P94=2.316228434547583, P95=1.3570235151475398, P96=0.15328395499967068, P97=0.016053411323072215, P98=0.016100756139521655, P99=0.007842963888839151, P100=0.016912226713202104, P101=0.21525345853008193, P102=0.9927809303993791, P103=0.5381000292397992, P104=0.04007146120064242, P105=0.007548086212750926, P106=0.016271722914885406, P107=0.0162372739613446, P108=0.08388426164474423, P109=0.3459789692514137, P110=0.2877171931651363, P111=0.10557462714453045, P112=0.02211015743494047, P113=0.009531220473994491, P114=0.06527744445380648, P115=0.09296272225429246, P116=0.015417774011822916, P117=0.009154947972816121, P118=0.015970105565378607, P119=0.0776548263418238, P120=0.030403322633684123, P121=0.0047066707027244034, P122=0.005980715690624752, P123=0.012753799952789241, P124=0.005593898092029201, P125=0.002175928526960374, P126=0.0030205970996223016, P127=0.004274726460606598, P128=0.002293989754758286, P129=0.0039208502814777205, P130=0.014759260109645544, P131=0.0090640491832837, P132=0.011087058565639836, P133=0.031601395246872964, P134=0.013779528709804652, P135=0.010335530878648812, P136=0.012657054849293808, P137=0.003610842051622762, P138=0.003539470647649903, P139=0.0024329850007056815, P140=0.0038967526734977618, P141=0.0048674701661598424, P142=0.007221121437127289, P143=0.004793840453668679, P144=0.005659870019360954, P145=0.00418247488815577, P146=0.002946558350510267, P147=0.0015665614384210904, P148=0.0007836146106694482, P149=0.000996417730231491, P150=0.0012460856753561485, P151=0.0014493062672425595, P152=0.001742053951033133, P153=0.002383156156314752, P154=0.001243625168299596, P155=0.0009514379042717612, P156=0.00142939446583863, P157=0.001238434916908422, P158=0.001319801924531489, P159=0.001010403821944701, P160=0.0009129009353549846, P161=0.0006045165213525293, P162=0.00037283381185983716, P163=0.00046796045793196, P164=0.0003694292615094386, P165=0.00035471322996653333, P166=0.0004900273597720086, P167=0.000392768981106995, P168=0.00034463711928662925, P169=0.00024203811510469287, P170=0.000178526869267761, P171=0.00024607633336971646, P172=0.0003491372611029196, P173=0.0004310860894043542, P174=0.00047730598644683136, P175=0.0006987208993792585, P176=0.00044258162757763363, P177=0.0005282849893298758, P178=0.0003551067443751156, P179=3.608256068950813e-05),Row( P0=-513.6927003734478, P1=51.6972683635212, P2=-4.91392477592844, P3=-0.8844811272591384, P4=-5.3228850343534715, P5=-3.4243435513762175, P6=-4.590051487225943, P7=-1.6662709913224425, P8=0.3184088596316972, P9=2.738256883635969, P10=-5.172699876605942, P11=-3.8142472380523587, P12=2.6078717824972415, P13=-0.704135811377419, P14=2.4783435567483365, P15=4.903620693337824, P16=8.224379123931994, P17=8.159127260970903, P18=7.289568681477668, P19=9.623131364432115, P20=6.982772075115302, P21=9.527251936450545, P22=6.047371762169737, P23=4.621918197491426, P24=1.5752963913279259, P25=3.5241431056677106, P26=-1.969987826612462, P27=3.2516229785935953, P28=2.4109660026343054, P29=4.454989088498656, P30=0.5776622390530501, P31=2.1537088594061737, P32=-0.40597268523034985, P33=4.2602984966464135, P34=1.2103188469931423, P35=2.4257455010652045, P36=-0.17283062709764094, P37=0.26391972122474927, P38=-0.22194370026813345, P39=2.1176506417930336, P40=0.6103829018064376, P41=0.5574820125211271, P42=0.6187922159210044, P43=0.6126549020090171, P44=0.6343656032878121, P45=0.6427652544617067, P46=0.6524747566587592, P47=0.6168811109811791, P48=0.5710450161700592, P49=0.5526240622213789, P50=0.5634974545517712, P51=0.6140315841414182, P52=0.9392005845228251, P53=0.2277165745645916, P54=0.07102093737580882, P55=0.02192403173658237, P56=0.003681987124602284, P57=0.0007098315770158518, P58=0.0012076437503256094, P59=0.0039023783876525314, P60=0.01581720196388496, P61=0.07017811078835479, P62=0.13386553135085505, P63=0.23029749078736586, P64=0.3382628684352604, P65=0.3961025565202618, P66=0.6559177666788399, P67=1.0614107472715397, P68=0.6013987013484494, P69=0.7055122148738152, P70=0.35862966025173754, P71=0.03909992179319964, P72=0.009446889861258656, P73=0.01027461259558749, P74=0.030098301862091288, P75=0.07704145105613631, P76=0.09463473378748334, P77=0.3950409403887629, P78=1.1582388173158142, P79=0.7025718938699909, P80=0.09225433408034317, P81=0.0407447881063554, P82=0.05416252618036949, P83=0.08214141193361355, P84=0.13003015893958156, P85=0.1185173270186736, P86=0.10615475356634424, P87=0.21370095484238077, P88=0.19586755162072508, P89=0.11540191378171445, P90=0.15530007231496054, P91=0.11734722895256772, P92=0.027276635926696456, P93=0.03358765261728196, P94=0.07524344189586032, P95=0.15689473637098386, P96=0.3945266569079668, P97=0.2601053365916061, P98=0.10862206273031756, P99=0.07822012210176034, P100=0.015918390161464294, P101=0.009061511174511556, P102=0.018029980069077228, P103=0.04588678714295979, P104=0.118516469165787, P105=0.781053561125887, P106=0.38204637912709205, P107=0.007030621815129777, P108=0.010346574695129665, P109=0.027619604790138547, P110=0.058453923123656146, P111=0.09358156030682288, P112=0.15824783148537688, P113=0.04697164722309356, P114=0.018384035601851586, P115=0.02636718768490475, P116=0.010404956121052062, P117=0.005588716401058398, P118=0.01209690657022357, P119=0.004855272221702616, P120=0.0015432510504889802, P121=0.005611990311436559, P122=0.0027019122368174904, P123=0.0008957894642682853, P124=0.0012744818127963676, P125=0.0027918452523226483, P126=0.0030343021817365763, P127=0.0026199599563672144, P128=0.0030668346556022024, P129=0.003544159138423673, P130=0.0074031810255674275, P131=0.0026855883520047127, P132=0.0024610624442642838, P133=0.009502210442502497, P134=0.0031795749513763446, P135=0.0019879281563760154, P136=0.006012524649017513, P137=0.0038506882176102164, P138=0.0027555721482971254, P139=0.0020647525649833635, P140=0.0009375349922064387, P141=0.0010263382320739513, P142=0.000981352605897508, P143=0.001185594066882836, P144=0.0012333366900029446, P145=0.0009164976555490258, P146=0.0008554735748075057, P147=0.0006776813284878452, P148=0.0002993571640179272, P149=0.0003753648338937709, P150=0.0006142587145212764, P151=0.00042655702772139586, P152=0.0006875658973180378, P153=0.0008192223682025636, P154=0.0006955342541646249, P155=0.0010979276585548563, P156=0.0007848505259393332, P157=0.0007153963011566224, P158=0.0007217329871349748, P159=0.0007029040822577653, P160=0.0005207678434433343, P161=0.00046864076472747854, P162=0.000624750058680884, P163=0.0005250831966117308, P164=0.0004212301233431268, P165=0.00032869528666165784, P166=0.00029500052850539127, P167=0.00025260580746909103, P168=0.00023398505244647861, P169=0.00019502298164279685, P170=0.0001854954488783833, P171=0.00023847320561331606, P172=0.0001776177336922403, P173=0.00025044405588891383, P174=0.0002782186699509067, P175=0.0003698303713365086, P176=0.000260369736454687, P177=0.00026817899957578135, P178=0.00012395584870362393, P179=1.1524157185993009e-05),Row( P0=-741.6365298578331, P1=73.92653595276226, P2=23.325486129791944, P3=17.35888907953789, P4=7.314920935188594, P5=3.8922181773078584, P6=4.446716188788059, P7=5.377011073209351, P8=2.7501282304871553, P9=2.240070631720903, P10=-4.266487667420496, P11=-0.3858807248675324, P12=1.4331561754350282, P13=-3.4067688596726238, P14=-2.8893008560857782, P15=1.6099325398975053, P16=0.20909858698793365, P17=-2.151166933061952, P18=0.8344717849131313, P19=0.667987920296694, P20=-1.536854824754189, P21=-1.9867170115315231, P22=-3.9132112377920496, P23=-0.3223437542692421, P24=-5.019001349593976, P25=1.0977517884830617, P26=-5.222503030334197, P27=-1.2205308122543923, P28=-0.5018484749306288, P29=1.2410232617178631, P30=-0.23110995125625985, P31=5.834828587845663, P32=2.146169715591513, P33=5.95182607365185, P34=2.1044172562986745, P35=4.597844063216597, P36=2.0033154569255633, P37=3.399677246540205, P38=1.697632352544321, P39=5.508168608285761, P40=0.6793047699971836, P41=0.6350867862508801, P42=0.6134523001478381, P43=0.6542148704799772, P44=0.6831981318702559, P45=0.6913582505766384, P46=0.7362058477757338, P47=0.7266100648784609, P48=0.7148636858678645, P49=0.7259175477464437, P50=0.7366117609557978, P51=0.7192092396505555, P52=8.185024392158013e-05, P53=1.1914561809612981e-05, P54=1.4667769522760235e-05, P55=1.241323079574684e-05, P56=0.0005302227144768549, P57=0.004441351465602777, P58=0.005441273007234332, P59=0.008696666968366492, P60=0.005234337758534367, P61=0.0005846497649164276, P62=0.0005181462468339945, P63=0.001178120673683073, P64=0.0016297350719354081, P65=0.004334402592270251, P66=0.007322791811696502, P67=0.006021447188148229, P68=0.005210325647134279, P69=0.0015702659377621238, P70=0.00038598856276713533, P71=0.0002792611748272195, P72=0.0003793993586940368, P73=0.0003491590203000141, P74=0.0005809009337209393, P75=0.0007746468652361959, P76=0.0009204138009722083, P77=0.0009334112057584242, P78=0.00048116078551380993, P79=0.00039763090344913934, P80=0.0003186070235431434, P81=0.0004742659359085636, P82=0.0006843841385083099, P83=0.0004045321254597106, P84=0.0006169816703106903, P85=0.0007374404849960259, P86=0.00017596177064603064, P87=6.243669299153297e-05, P88=8.333540935332935e-05, P89=8.568616864965162e-05, P90=0.00045690147552467623, P91=0.001386529952628669, P92=0.0006286571117831796, P93=8.070812413129459e-05, P94=5.2513701671487654e-05, P95=8.872305499459389e-05, P96=0.00028607760886333484, P97=0.00023652095078069297, P98=5.199487728776514e-05, P99=1.88293183869122e-05, P100=2.6355262092821022e-05, P101=0.00019374584663703527, P102=0.00010496477726070565, P103=2.514498485635548e-05, P104=1.3452829252863831e-05, P105=4.326316379130556e-05, P106=4.1113756770507944e-05, P107=1.820072318044455e-05, P108=3.3116390014359614e-05, P109=6.927374115797368e-05, P110=2.1732812910217545e-05, P111=1.7313842127454408e-05, P112=1.9426665066537043e-05, P113=2.4223409647282216e-05, P114=2.23593481866928e-05, P115=3.941220582564123e-05, P116=3.668678240256831e-05, P117=3.305122055557349e-05, P118=2.5192452190985047e-05, P119=2.8358114116569234e-05, P120=2.442442145443877e-05, P121=2.947117375656156e-05, P122=1.5252335587269078e-05, P123=1.25418636624266e-05, P124=1.3253748123682763e-05, P125=1.0125233169843483e-05, P126=2.902619892229693e-06, P127=5.261098621791572e-06, P128=3.970763733711458e-06, P129=2.2895288648514692e-06, P130=5.339273513659867e-06, P131=5.98323047267961e-06, P132=6.424342721233463e-06, P133=1.5266267633900718e-05, P134=1.3870459626812822e-05, P135=1.2884472255266736e-05, P136=1.1003778379002566e-05, P137=7.135918257559064e-06, P138=6.2456504137386294e-06, P139=5.883779601059505e-06, P140=6.293946295467547e-06, P141=9.351262030005535e-06, P142=1.5819705686103485e-05, P143=1.8411550045310654e-05, P144=2.4447100171460287e-05, P145=2.0318728651544847e-05, P146=7.646526848345784e-06, P147=6.426872470277414e-06, P148=6.530813865287492e-06, P149=1.792090314979401e-05, P150=4.357050332981904e-05, P151=6.345182496298176e-05, P152=5.999638841625004e-05, P153=0.00010690502829633772, P154=9.610450279210141e-05, P155=7.324138883079993e-05, P156=4.4103018052909876e-05, P157=6.542206617560366e-05, P158=5.868545310119185e-05, P159=6.042703645322264e-05, P160=5.0209733063587493e-05, P161=4.075489518254537e-05, P162=3.4287905380364835e-05, P163=2.261768420884397e-05, P164=1.9683238720142843e-05, P165=3.149786689950968e-05, P166=2.4771406661582345e-05, P167=2.2206215381877444e-05, P168=1.9619756607439146e-05, P169=1.34493081508489e-05, P170=8.147665652655924e-06, P171=1.1260636222235022e-05, P172=9.514741451848168e-06, P173=1.1716325308644157e-05, P174=1.4582986595225796e-05, P175=1.5466317923198896e-05, P176=1.7826209575778578e-05, P177=1.5916874368580538e-05, P178=6.938431130978138e-06, P179=7.332510812010242e-07),Row( P0=-472.17716013573784, P1=26.335066250249362, P2=-16.754239775695925, P3=-3.5504974764180965, P4=-11.867130520463242, P5=-10.061998071936012, P6=-7.035460551836942, P7=-6.330095920689636, P8=-0.5226797827638014, P9=-2.7796379906671436, P10=-8.30039711148836, P11=-2.0263964898448568, P12=4.35330525103339, P13=5.858810456478738, P14=7.999841535896425, P15=13.347106480793219, P16=10.417856098781492, P17=5.8531660398426135, P18=0.7846731597618494, P19=6.144865013584143, P20=0.3994052174330732, P21=3.269466533673011, P22=1.2857783126476134, P23=1.1542400347475241, P24=-1.116623981659788, P25=1.8836375785653754, P26=3.3809790478707384, P27=1.8912315469628185, P28=1.4994455068919519, P29=4.697074832358453, P30=0.8223398114913012, P31=3.9093596059823175, P32=-1.5936279973277931, P33=3.6992910626493716, P34=1.6881245859432485, P35=0.8933964839773275, P36=-0.7996925395252027, P37=1.25927373339389, P38=2.348087375982825, P39=0.514715247963256, P40=0.6208619338130164, P41=0.6579938976993261, P42=0.6624035763920109, P43=0.6613937201806656, P44=0.6774458049149459, P45=0.6981094373048292, P46=0.6714117909556839, P47=0.6278890597817971, P48=0.6470319213672525, P49=0.6527694889643776, P50=0.5909127061992419, P51=0.6264579473997328, P52=0.0162026861835047, P53=0.0027911238657537846, P54=0.0011618719104260881, P55=0.0002956536709622133, P56=0.00022862934488366532, P57=0.0002472864058046427, P58=0.0002898470498490303, P59=0.0018444117079133718, P60=0.006986351823505499, P61=0.01513819636707615, P62=0.032888930355751834, P63=0.06138490768249169, P64=0.07749809549479203, P65=0.17788532639969915, P66=0.41714463198654633, P67=0.5698790813019439, P68=1.2058631256353045, P69=1.6383747446173837, P70=0.9883581315290523, P71=0.9782584992495515, P72=0.9538888135602954, P73=0.34616671120099873, P74=0.0711242424637496, P75=0.04628561051578661, P76=0.028984431403282277, P77=0.054997644247171976, P78=0.081790603358108, P79=0.12496337218447402, P80=0.18853118418065368, P81=0.279761770794088, P82=0.5093575560333664, P83=0.7240132245841555, P84=1.675625916768093, P85=3.190369761105867, P86=1.3767909024509848, P87=0.7383907796602519, P88=0.4229778325850762, P89=0.4112808031628157, P90=1.9868818340421248, P91=3.8110005482328755, P92=1.2476341030967166, P93=0.2223231491307177, P94=0.3778536948526728, P95=0.43091917303847604, P96=0.26608675880791377, P97=0.7185202213925064, P98=1.452748704189081, P99=0.20592169370846275, P100=0.019863136134196716, P101=0.06957983688402454, P102=0.7556632187992512, P103=2.383799549034717, P104=2.7972849640299904, P105=0.24648961295602034, P106=0.10889908034406294, P107=0.09765250221669845, P108=0.028802738383066576, P109=0.026548879607549133, P110=0.2594002044484522, P111=0.773159912626916, P112=0.5372478206998909, P113=0.5125336613530141, P114=0.12372586756985725, P115=0.01867678851788838, P116=0.00794619353614946, P117=0.006422882462115293, P118=0.028409509958025943, P119=0.2504175128298096, P120=0.1869910463170241, P121=0.008840427514484193, P122=0.0017961112541542744, P123=0.011795724334320348, P124=0.05280519015510999, P125=0.0197187702168845, P126=0.002793988647208573, P127=0.0047924762301960145, P128=0.01103373307367592, P129=0.007313960072418374, P130=0.006298829816553933, P131=0.01701638717746346, P132=0.04112891901838527, P133=0.054600101401773043, P134=0.02929312345623348, P135=0.024817843543999248, P136=0.03664861961132971, P137=0.03683721257681054, P138=0.017050846947522586, P139=0.006209514619117616, P140=0.01181586968255521, P141=0.011754651334556646, P142=0.009444912535901368, P143=0.016037361658124718, P144=0.010950424234469023, P145=0.017510567214506112, P146=0.008438575606576506, P147=0.005937760521097291, P148=0.005089390537743465, P149=0.0048935409542134, P150=0.005913942797578598, P151=0.008496050867622978, P152=0.007119516105328483, P153=0.012515004892729149, P154=0.007120790670453963, P155=0.0068746822900649625, P156=0.006564228192003892, P157=0.009535739434961514, P158=0.007303792259656201, P159=0.007357422103573857, P160=0.005633786832407662, P161=0.0036179709671169736, P162=0.002585212064080693, P163=0.0022921671795431706, P164=0.001970641746840693, P165=0.0022699540634347694, P166=0.0028845263130081182, P167=0.0027382608092382243, P168=0.003378394421950209, P169=0.003053665810268747, P170=0.003670321851071918, P171=0.0027380979250661063, P172=0.005257963065778968, P173=0.005047692977329916, P174=0.005693293418002466, P175=0.00632266361171763, P176=0.009075585070586601, P177=0.009719964374922254, P178=0.0050955309610281945, P179=0.0006704637788335757),Row( P0=-554.1483940635075, P1=28.246117833209198, P2=-20.13968001568174, P3=-10.526488325185356, P4=-16.145572318675242, P5=-10.648468782899506, P6=-15.54060228350725, P7=0.7026608086056644, P8=-9.184849318647837, P9=-5.48496244732226, P10=-6.281647754751811, P11=-1.620793373599708, P12=2.4441481378744405, P13=-0.9982531837554981, P14=5.762889958868296, P15=4.40478782800215, P16=8.260389007441786, P17=9.755715720626045, P18=2.7411141578355713, P19=5.353595341465158, P20=-4.4009122632572435, P21=1.6949015950120059, P22=3.6303629863981994, P23=2.203534074631144, P24=1.197460481824552, P25=-0.12617271287754916, P26=-1.9837090594445035, P27=1.2620449649368735, P28=2.181076539424563, P29=5.056054371842978, P30=0.8186068819238504, P31=3.8187864029911527, P32=1.8869350809235754, P33=5.342815893312606, P34=3.965939030686711, P35=2.691053218789258, P36=-0.952968493130054, P37=0.8629546713166666, P38=1.6800672042012508, P39=2.0950124389877973, P40=0.7223209380733586, P41=0.6402906313790321, P42=0.6494581045876164, P43=0.6593111179052543, P44=0.640610548085849, P45=0.6553840747496141, P46=0.6801292148207346, P47=0.6825283708613203, P48=0.6483329911277216, P49=0.6560971238755295, P50=0.7129814021495862, P51=0.7716210623992875, P52=0.00019353642413824998, P53=5.512348874819563e-05, P54=5.207336309425006e-06, P55=7.664306504033886e-06, P56=3.6486979432257366e-05, P57=0.0002905925488815047, P58=0.0008420502010385827, P59=0.002283417271214405, P60=0.0021180289183442298, P61=0.005681974857510394, P62=0.008794494278500075, P63=0.008011474243420398, P64=0.005102462660909254, P65=0.015227954635349138, P66=0.08179689208633215, P67=0.23731250856199526, P68=0.21436937128425315, P69=0.09495952520283509, P70=0.09496458922738649, P71=0.04839009114515717, P72=0.01839001361589594, P73=0.015193237117161945, P74=0.013480636472975304, P75=0.016435325286588483, P76=0.020559301564298846, P77=0.05124778697858542, P78=0.0474084084927567, P79=0.0363959466539791, P80=0.019541186137241384, P81=0.024891997599864622, P82=0.06363253947283075, P83=0.1416586661923106, P84=0.22502396679522485, P85=0.3111949247732473, P86=0.14239853575706474, P87=0.08800982895386009, P88=0.474642095512239, P89=0.42340154074204484, P90=0.16443314442165358, P91=0.19252993060249846, P92=0.44340439803621734, P93=0.32812766533716836, P94=0.279640888772222, P95=0.38972273831371246, P96=0.4023697870461269, P97=0.43198854811955983, P98=0.3866654534666302, P99=0.20135589235450818, P100=0.06142045627672079, P101=0.028809158870831995, P102=0.01711500526232268, P103=0.017723482370852757, P104=0.05787423734864756, P105=0.04543590681308023, P106=0.034809567565445496, P107=0.027319040506187695, P108=0.019653445820344933, P109=0.023582434355567623, P110=0.05361300195790453, P111=0.12380858230395811, P112=0.05041511007748699, P113=0.018124816628479253, P114=0.007794778799005804, P115=0.0065134809354698234, P116=0.05157430816865117, P117=0.09636069112387263, P118=0.009110838315784109, P119=0.005454177808913156, P120=0.003872031712262121, P121=0.010610548908471294, P122=0.018701689467819343, P123=0.01985230518870206, P124=0.03050506034165694, P125=0.005416800074266942, P126=0.0034529995304789407, P127=0.0026389563602874817, P128=0.006014306726803322, P129=0.005720011297578872, P130=0.002798545704950972, P131=0.0035479676503808407, P132=0.00932518081808641, P133=0.0041124878478943005, P134=0.002658809352497545, P135=0.002974993544807383, P136=0.0016192057812762762, P137=0.0008315960248589315, P138=0.0007783521760076176, P139=0.0003857288735548058, P140=0.0005637123851332769, P141=0.00046307591554239696, P142=0.000486099342938915, P143=0.0008403536693683398, P144=0.0008113243785303996, P145=0.0008821890466492326, P146=0.000949203632797804, P147=0.0021920899087251975, P148=0.000999584170496885, P149=0.0005384474532819916, P150=0.0015112702205505428, P151=0.0006863668486655634, P152=0.00021198044500537214, P153=0.0002989186627991888, P154=0.00037478250064355877, P155=0.00044849580183690306, P156=0.0009134483348530776, P157=0.0017894551507881451, P158=0.0028399823218445445, P159=0.002985349304633667, P160=0.0015987203122189441, P161=0.0010906473988579245, P162=0.000568105093713054, P163=0.00041366972930341196, P164=0.0002675085575647604, P165=0.0002520046830978212, P166=0.0002110842657300483, P167=0.00019365401779071074, P168=0.00017751964177452196, P169=0.00013679939949311071, P170=0.00010623085775614815, P171=7.226019353049918e-05, P172=8.484104735557731e-05, P173=0.00017052460307130288, P174=0.0001649660530118677, P175=0.00016424333601033108, P176=0.00018326225234855382, P177=0.00018138416539925878, P178=5.786301605389619e-05, P179=8.711670536127503e-06),Row( P0=-739.5407510330709, P1=74.77877007001337, P2=5.755184269670474, P3=6.89179219609065, P4=-0.0846743707296158, P5=-4.98035056775213, P6=-0.8571238819335785, P7=3.5475669519657247, P8=6.01389359997638, P9=0.4629674867613097, P10=-3.1774341210447115, P11=2.203055677962131, P12=-0.8985737186443326, P13=-1.31111988767444, P14=5.512257412678029, P15=-1.9337420774626177, P16=0.7483542711429336, P17=3.623079721837965, P18=-2.9065572268852313, P19=4.06125269953548, P20=-3.6189597761931096, P21=-0.7906724498234664, P22=0.01750486685005872, P23=-2.052801647814191, P24=-1.5569126736472523, P25=-0.2591366121094056, P26=-1.0985218847790081, P27=-0.820051408650829, P28=-0.997131764795275, P29=-0.7239768526568436, P30=-1.0537222103929365, P31=0.7178182319608867, P32=1.3305014945069602, P33=2.9045475714449362, P34=5.025973538403961, P35=4.078054568294883, P36=3.248684618271639, P37=4.030998499291411, P38=4.062064907451179, P39=2.4700586888188756, P40=0.7658017317312549, P41=0.6909961801158827, P42=0.6861898782988014, P43=0.7238568406201793, P44=0.746695702666467, P45=0.7038845043793489, P46=0.7060689671396667, P47=0.6687568292229317, P48=0.7070613662911642, P49=0.7604772340037532, P50=0.7862532875099499, P51=0.8025695047103109, P52=2.431234776842397e-05, P53=1.2728906189110123e-05, P54=7.908814798811353e-06, P55=5.626276693104569e-05, P56=0.0003085623514410229, P57=0.0022659376832917947, P58=0.004896123869217272, P59=0.004415663842783346, P60=0.0016335131872849322, P61=0.0007031078383637216, P62=0.00038916244927234196, P63=0.0005678454452620824, P64=0.0016619948937372244, P65=0.005496766103413761, P66=0.007931945790825559, P67=0.00587329448002298, P68=0.0019127538572334558, P69=0.0018240360609929661, P70=0.0032804208450240625, P71=0.002021904232321651, P72=0.0018864235773097269, P73=0.0035521573954523386, P74=0.0035824659618902016, P75=0.0019508876559741733, P76=0.0012765905776664288, P77=0.003129293310466331, P78=0.007049594447047086, P79=0.012841974491178621, P80=0.013847853716745708, P81=0.011093678959500865, P82=0.008097172966920493, P83=0.0031388541120615974, P84=0.0030956576243299162, P85=0.0031848613811975984, P86=0.0029999331814539384, P87=0.003966112098567972, P88=0.003936210342948291, P89=0.0025539845808610815, P90=0.0044791634595808694, P91=0.0033977357045753033, P92=0.0011580926315546735, P93=0.0006079974239262389, P94=0.002688264660457894, P95=0.0016828906646632238, P96=0.0003733602412683481, P97=0.0003230001543342211, P98=0.0003926801879290949, P99=0.0005115046811207103, P100=0.0001669228394519428, P101=5.1666391885594386e-05, P102=4.6798234102536186e-05, P103=6.616047942157261e-05, P104=6.841847198442979e-05, P105=0.00020255125105079274, P106=0.0002546686267450061, P107=0.0002619630866690742, P108=0.00024415864509997774, P109=8.163635536402944e-05, P110=0.0003441512917061293, P111=0.0005604964225167193, P112=0.00032441061067372683, P113=0.00015680286071043169, P114=6.700694288979986e-05, P115=5.6534278326084345e-05, P116=0.0001520034457848751, P117=7.68790720376049e-05, P118=0.00018542970123910447, P119=0.00016055360316280542, P120=0.00024471572713989723, P121=0.00037973383365925533, P122=0.0005323331378859327, P123=0.00017628413735335937, P124=9.055887495873148e-05, P125=9.387364962482255e-05, P126=9.216778670211262e-05, P127=5.6606434289707007e-05, P128=2.8393492960926342e-05, P129=1.3995418416756069e-05, P130=1.4142394416989979e-05, P131=1.1960056187213787e-05, P132=1.1232128447664943e-05, P133=1.774635302997592e-05, P134=3.927162931723753e-05, P135=3.8182051628341665e-05, P136=3.028558588645801e-05, P137=7.590463419314665e-06, P138=4.783694486788732e-06, P139=8.741710476122165e-06, P140=6.107637698844927e-06, P141=1.1342647113220767e-05, P142=2.1232365543670335e-05, P143=4.9831324429482646e-05, P144=6.58104387959959e-05, P145=3.8562073087972755e-05, P146=2.1724261448935866e-05, P147=1.9934059158265003e-05, P148=8.46931159131496e-06, P149=3.1333441973566486e-06, P150=2.6618201850016296e-06, P151=2.955136322740476e-06, P152=3.855694996025657e-06, P153=9.890166712446996e-06, P154=1.265278001553426e-05, P155=2.2632097284466015e-05, P156=1.7544600469590446e-05, P157=1.0611476736673602e-05, P158=1.541388811479558e-05, P159=1.2267827133645807e-05, P160=1.0209253242006453e-05, P161=7.474191088123199e-06, P162=5.855794276978285e-06, P163=4.581485361138314e-06, P164=7.700754905167747e-06, P165=6.525636349859868e-06, P166=5.267139151777405e-06, P167=5.548690024104517e-06, P168=4.6172033829168185e-06, P169=2.584849656725203e-06, P170=5.0533650320728355e-06, P171=4.144200544357117e-06, P172=3.960542309933454e-06, P173=3.132319329542455e-06, P174=3.3633993763017247e-06, P175=4.5557232423080046e-06, P176=5.2743417693802905e-06, P177=6.157002512623203e-06, P178=1.8442940626658466e-06, P179=1.946723827957633e-07),Row( P0=-729.2773887975195, P1=61.244151655737085, P2=-4.462093567621715, P3=8.315630465106196, P4=-2.3467779098301262, P5=-6.224961024205285, P6=-0.05027368867157148, P7=3.442162857474087, P8=6.798057644199117, P9=-0.955646767142202, P10=-5.814329038932114, P11=2.506970269688889, P12=1.4866479717748087, P13=-0.6667222907641951, P14=4.064106244682573, P15=-2.2956677535911627, P16=1.8617261245404115, P17=2.652980659918393, P18=-3.4659713562341983, P19=4.172889889299145, P20=-3.156784805359702, P21=-1.8670403487886573, P22=0.2850258054699626, P23=-3.277977692840132, P24=-2.388523098281412, P25=-0.42373277413498656, P26=-1.5923086385271503, P27=2.077004951692989, P28=1.3279495605619496, P29=1.9649646919080825, P30=2.4549853250439737, P31=2.4872115245869377, P32=1.6036747028701148, P33=2.2244734975724305, P34=2.5226163435595037, P35=3.7958294578647074, P36=2.8170674766122135, P37=4.136059436789644, P38=4.508061689715594, P39=2.6681428478536455, P40=0.7210532471562915, P41=0.6945155743073166, P42=0.7308221396785528, P43=0.732493765578266, P44=0.7406924226789479, P45=0.7796411095293713, P46=0.706683473505089, P47=0.6873541127439877, P48=0.6829563879826493, P49=0.7079236090165102, P50=0.7382774259975768, P51=0.7300101320631386, P52=1.739148836194124e-05, P53=8.925545241446082e-06, P54=1.913143105839901e-05, P55=3.783734791101951e-05, P56=4.653671596451183e-05, P57=0.0005814739154305164, P58=0.0023779447256963896, P59=0.0035465734011102526, P60=0.0033863073766607346, P61=0.0018384340735219315, P62=0.0006040831890226005, P63=0.00036753473222225135, P64=0.000379726913282986, P65=0.004804207606854621, P66=0.007860607571656682, P67=0.006984869548917259, P68=0.004291230503091257, P69=0.004183078377789087, P70=0.0064542167825529335, P71=0.002452076281431347, P72=0.0011694617897734696, P73=0.002082467998100017, P74=0.0018861068728961833, P75=0.002553895810159134, P76=0.003031688474194693, P77=0.004281236666372724, P78=0.0064907260726989845, P79=0.009618048125603922, P80=0.00885599690116405, P81=0.0025283851810963586, P82=0.0016387137590300352, P83=0.0016556238250124678, P84=0.00233609133609092, P85=0.002973201885196879, P86=0.008635470035869966, P87=0.014687003045247518, P88=0.019992362016438805, P89=0.007048849207917319, P90=0.0009880671784598884, P91=0.0025814732542660313, P92=0.021486598859216634, P93=0.011072839025345407, P94=0.015648915375330686, P95=0.023992148998429014, P96=0.005335263578559986, P97=0.0017360944720556562, P98=0.0003873458750700675, P99=0.00020831322328240616, P100=0.0002872605074112461, P101=0.00033736402936143406, P102=6.0239202026931504e-05, P103=2.4357683238874925e-05, P104=0.00014503854871236052, P105=0.00044637799187691174, P106=0.0004534118662059613, P107=7.763198654797046e-05, P108=0.0001803807919240942, P109=0.0013318743159009211, P110=0.0035859710915931856, P111=0.0018057032351064997, P112=0.0013939606751080347, P113=0.0010906764321768063, P114=0.0008480462065395301, P115=0.00040666017195550597, P116=0.0010586211881753844, P117=0.0004867542751974501, P118=0.00042822332606286923, P119=0.00025635647724032626, P120=0.0004485846291060348, P121=0.00031118056085820015, P122=0.0001801181565815478, P123=0.0001312901872540726, P124=7.503489054013477e-05, P125=7.917866653408962e-05, P126=9.332841606604145e-05, P127=9.881801528965916e-05, P128=6.840386137283358e-05, P129=9.948899840128337e-05, P130=5.582657161496338e-05, P131=8.40348926588797e-05, P132=9.475648465836857e-05, P133=8.51500248158905e-05, P134=6.277750274850134e-05, P135=4.563588764117149e-05, P136=1.8991539165090932e-05, P137=1.2616064266982106e-05, P138=1.624735392092259e-05, P139=1.0375866762081806e-05, P140=1.4640034804231155e-05, P141=1.7693264646547578e-05, P142=3.46974515564133e-05, P143=7.977104820985515e-05, P144=0.000174204982693481, P145=0.00030921990998473737, P146=0.00020653045903997215, P147=9.019049148306482e-05, P148=1.7368097981680038e-05, P149=5.832628559580617e-06, P150=3.7974185523750105e-06, P151=1.0011948823182632e-05, P152=1.827048802015207e-05, P153=3.086905877276619e-05, P154=4.409456474271541e-05, P155=2.676893635219324e-05, P156=2.285554203705298e-05, P157=2.5534312434744505e-05, P158=1.9117144575704653e-05, P159=1.1225010614886667e-05, P160=5.512024861680744e-06, P161=5.667446350771886e-06, P162=8.422613200227943e-06, P163=6.533505628807702e-06, P164=4.532754767424049e-06, P165=3.966326630040786e-06, P166=3.3593629552462553e-06, P167=2.9484481001412296e-06, P168=3.4779398916905594e-06, P169=4.262270893412746e-06, P170=3.70463859177632e-06, P171=1.7044728667232708e-06, P172=1.3853774269165246e-06, P173=2.252918417064502e-06, P174=5.128799047723709e-06, P175=8.191265349067045e-06, P176=5.551020203925768e-06, P177=3.517606927018728e-06, P178=2.0733508417302807e-06, P179=1.8747862110272247e-07),Row( P0=-749.9886236095941, P1=73.02497038652056, P2=-2.1786056139887195, P3=9.94426460718252, P4=-4.717691960071266, P5=-0.5859549420399139, P6=2.776812243776313, P7=4.5683870201471715, P8=4.420980919717068, P9=-0.6267202440610231, P10=-3.6413390865948974, P11=-0.40741252351560403, P12=-2.01535442019282, P13=-2.0028087034589013, P14=2.0700470082926588, P15=-2.7020890823751738, P16=1.9355086978746618, P17=0.8074899066872021, P18=-1.1968176177067316, P19=3.3774873196463626, P20=-4.027081547013502, P21=1.7027156889475665, P22=0.17023530261540032, P23=1.3753361275565699, P24=-1.088153368269096, P25=0.6736070270277571, P26=2.435171227834533, P27=0.15731802817338367, P28=0.80055229915317, P29=2.2069496684179755, P30=0.837507042310366, P31=3.310008665506097, P32=-0.004062670130703297, P33=0.33832270414490234, P34=-0.2085136127074254, P35=-0.2641551995951187, P36=0.2321029734912444, P37=1.8945099638483938, P38=1.3483245690942722, P39=3.25564604452826, P40=0.7581765966630793, P41=0.7512267693186372, P42=0.7627412939132591, P43=0.804146641695368, P44=0.7798546371371485, P45=0.7540407450820228, P46=0.782768670552438, P47=0.7416616550402313, P48=0.763155481849938, P49=0.7839017522495043, P50=0.7273729286755126, P51=0.7213224191702984, P52=8.022607712880404e-05, P53=2.0837554930515884e-05, P54=5.620171874051943e-06, P55=3.7601030128291404e-05, P56=0.0003714997375420261, P57=0.0038100819413473914, P58=0.004229692477277441, P59=0.002198797705684664, P60=0.0020305826711811254, P61=0.0012755134276870679, P62=0.003023796638779409, P63=0.004585289051245633, P64=0.0051311348345766915, P65=0.003109301054945381, P66=0.001549974720186675, P67=0.002097722456692278, P68=0.0014724562136580865, P69=0.0018357054297248204, P70=0.0024359261336015653, P71=0.002066627924857254, P72=0.000665263027412499, P73=0.0006907829061211142, P74=0.0013930183295333036, P75=0.0020958596758430784, P76=0.001823097921514124, P77=0.004360767984971838, P78=0.008446821931941404, P79=0.005497598515876623, P80=0.001383335354396692, P81=0.0009926478364560495, P82=0.0027385193602157474, P83=0.006500856729764382, P84=0.009284129802974774, P85=0.008886099767444913, P86=0.006293393031642137, P87=0.006992513081196688, P88=0.006589016007183947, P89=0.006661615967959868, P90=0.008835319744431941, P91=0.0035139097150791183, P92=0.002657989682872554, P93=0.001953161799494613, P94=0.007327283278350758, P95=0.00656726875384574, P96=0.0011257998961551022, P97=0.0008313020369038707, P98=0.0022312472216594577, P99=0.003715699719765533, P100=0.004279373270479639, P101=0.002810864511659555, P102=0.0011527803681416892, P103=0.0008871207456590358, P104=0.0004090271640849559, P105=0.0006326714650859378, P106=0.0008375237309733017, P107=0.0005048313825514826, P108=0.0002757492483597513, P109=0.0006179295373744451, P110=0.0007748060514922774, P111=0.001148606572040443, P112=0.0004912004657165384, P113=0.0002182493520996414, P114=0.00012507650354873543, P115=8.766921576796834e-05, P116=0.00015141741717973104, P117=9.80423643124672e-05, P118=0.00021634796654622956, P119=8.708276072715643e-05, P120=0.00022862454806886432, P121=0.00012666704885693603, P122=0.0001605985062150706, P123=0.00011997626947175842, P124=5.1018176589657995e-05, P125=5.6784362126797106e-05, P126=3.837488099103849e-05, P127=4.053954509659723e-05, P128=4.551689026150805e-05, P129=8.079864317453061e-05, P130=6.611834525371106e-05, P131=4.382875830463015e-05, P132=7.473858834722072e-05, P133=5.384081376357253e-05, P134=3.1880731050204035e-05, P135=2.02254520171824e-05, P136=1.786601593537368e-05, P137=6.695198879262037e-06, P138=1.3506646761208263e-05, P139=1.204477195414724e-05, P140=2.419459766339275e-05, P141=2.0325326489111228e-05, P142=2.1124248727937646e-05, P143=2.2290507748519606e-05, P144=5.056729877538126e-05, P145=6.954353341101923e-05, P146=4.051449825224518e-05, P147=1.2318611241326833e-05, P148=8.614462874293832e-06, P149=2.0157910856593692e-05, P150=8.874013125868189e-06, P151=5.434815055366875e-06, P152=2.837430548731188e-06, P153=1.632777545774988e-06, P154=1.8148259832501105e-06, P155=3.0911632020889515e-06, P156=4.304824200830271e-06, P157=8.961573085026492e-06, P158=7.366106331129766e-06, P159=7.821552014296681e-06, P160=6.040113519897404e-06, P161=4.424223666291609e-06, P162=2.4039217458198626e-06, P163=2.137270604810548e-06, P164=1.2523814369616088e-06, P165=8.291929348783226e-07, P166=1.0032406835875017e-06, P167=8.225537306163577e-07, P168=1.035972797811096e-06, P169=9.671595303081868e-07, P170=8.093862756097765e-07, P171=4.938619926560864e-07, P172=3.444204337371381e-07, P173=5.499522944862387e-07, P174=6.595500730003393e-07, P175=5.755827846457623e-07, P176=7.703850204859e-07, P177=7.272080699918613e-07, P178=5.096717480913977e-07, P179=7.201446720423751e-08),Row( P0=-687.6830518183621, P1=54.70565798849033, P2=-9.385343456635102, P3=3.5804444986134456, P4=-7.732727784528623, P5=-7.994861018178916, P6=-6.406150965833032, P7=3.8327839028744726, P8=0.5099755759370472, P9=-0.5098574378895281, P10=-7.244523620506692, P11=0.8537195013375841, P12=0.5070965326013763, P13=-1.617020318356156, P14=3.3422667061310185, P15=-4.3498703098151035, P16=1.3004177480519417, P17=0.9617011019775816, P18=-2.269809039472435, P19=4.293118953329809, P20=-3.953271781952907, P21=3.1139517260243528, P22=1.8940147916130883, P23=0.47747908545797957, P24=2.837606339971653, P25=1.6192787645912508, P26=2.4609693107037334, P27=4.559003100744776, P28=1.709561782037037, P29=4.193413865381869, P30=1.3781111274689086, P31=2.567515092284065, P32=1.4782297841642753, P33=1.2527863716042837, P34=3.044240357767573, P35=3.8096146864144145, P36=2.6181997862116875, P37=2.7578898790843813, P38=1.101040466933948, P39=0.2470162160073024, P40=0.6816405567425668, P41=0.685588475951026, P42=0.7298231435524776, P43=0.7484639933641676, P44=0.7437340071426232, P45=0.7555218610946344, P46=0.7269774458917507, P47=0.6714452214767276, P48=0.6610741481990413, P49=0.7036095846625328, P50=0.7133910384022125, P51=0.6921745835138795, P52=2.019504599462911e-05, P53=1.0532457948638415e-05, P54=4.473599369394194e-06, P55=1.0641404749486772e-05, P56=5.478424058482212e-05, P57=0.000568613261125148, P58=0.0018296713960373422, P59=0.0032738031734043356, P60=0.00596637392621412, P61=0.009488225554477052, P62=0.010544492025108532, P63=0.0077152475226612135, P64=0.0031258205451510914, P65=0.00306456327706259, P66=0.006451945127838064, P67=0.00433529442638364, P68=0.006275614728680338, P69=0.013401793011056387, P70=0.021010439762882412, P71=0.011094335864838597, P72=0.005703808668067515, P73=0.008924243964986716, P74=0.00819323841416633, P75=0.008425094882022715, P76=0.009756867268022357, P77=0.009883111123664351, P78=0.011999587209316853, P79=0.022455083460664875, P80=0.01906721424081083, P81=0.014498302484573214, P82=0.01121495617893404, P83=0.004842873293816944, P84=0.005993892926698093, P85=0.011287391918456792, P86=0.013891728529912387, P87=0.018911207141346746, P88=0.052176474751544175, P89=0.04705842342259703, P90=0.023081476438417448, P91=0.007793250322856384, P92=0.011210454464944087, P93=0.02058822271524475, P94=0.06598971876522478, P95=0.04105748553062068, P96=0.01328033413974875, P97=0.0029795159797672936, P98=0.0011143493314761284, P99=0.0013868191266490687, P100=0.0014491643023215469, P101=0.0013009296824189134, P102=0.0004957173845096516, P103=0.000318595142221732, P104=0.0002628540272306685, P105=0.0008997964609937214, P106=0.0022104352709085766, P107=0.0014408266082658171, P108=0.0005512377395710883, P109=0.0016240989497546686, P110=0.006888405606628117, P111=0.010755451996822851, P112=0.004205946799702864, P113=0.0017885603628819396, P114=0.0009974774744731735, P115=0.001746419464520715, P116=0.0018172488567693838, P117=0.0011959660583681694, P118=0.0024964836720657723, P119=0.00032197094582200045, P120=0.000508115866027452, P121=0.0006133267275338884, P122=0.0007944758379323914, P123=0.00023062406543991476, P124=0.0005963927151926387, P125=0.00048267912798288445, P126=0.00025412096516502455, P127=0.0003714401836053149, P128=0.00024045139850758327, P129=0.00041274322577708423, P130=0.00030382725314672635, P131=0.0001411540783062293, P132=0.00011559972521485298, P133=0.00019426118551675895, P134=0.0003128318754237889, P135=0.0001883480704645529, P136=0.00014933852329409667, P137=8.308420943385044e-05, P138=4.550245843865254e-05, P139=3.097292646966159e-05, P140=3.508540987232039e-05, P141=4.8327609387838374e-05, P142=5.087654369469142e-05, P143=7.825437690117367e-05, P144=0.0001579544897755112, P145=0.0002854958673541682, P146=0.00022770996406112437, P147=0.00019960711671803227, P148=0.00012944017287992016, P149=5.320729125330642e-05, P150=1.1433052049617922e-05, P151=2.9453110518477334e-06, P152=4.482102398090363e-06, P153=1.4058361725729283e-05, P154=2.910021441591431e-05, P155=4.688818134056525e-05, P156=6.05450055903209e-05, P157=5.154496578124205e-05, P158=7.075371683742139e-05, P159=7.715199685151075e-05, P160=6.826400662058668e-05, P161=5.6615753805324244e-05, P162=2.8659795958695935e-05, P163=1.8421463120934958e-05, P164=1.7436018406243093e-05, P165=1.8440487549156988e-05, P166=2.0170737867937145e-05, P167=1.5770257237272288e-05, P168=1.3045010068228584e-05, P169=9.260207234712895e-06, P170=6.2693794421887285e-06, P171=5.173780386909653e-06, P172=6.4632356323683305e-06, P173=6.333939866193564e-06, P174=8.630130938758402e-06, P175=9.63580867191287e-06, P176=5.741763939319661e-06, P177=9.583860017096818e-06, P178=6.8182346357151895e-06, P179=5.869634435554095e-07),Row( P0=-697.4277916498605, P1=58.484427716339404, P2=-2.1866573741856348, P3=4.18272286012835, P4=-4.440632652271294, P5=-6.726311821259674, P6=-1.3325025484496174, P7=4.552819798971609, P8=-1.5972133307441136, P9=1.4167620989983434, P10=-1.7950752380360087, P11=0.8132733898257017, P12=-0.3121751174354528, P13=-3.154013826000268, P14=3.253391499603687, P15=-0.3585006591775622, P16=0.23813073061725298, P17=1.4807492573784766, P18=-1.050873109424452, P19=2.666193249642857, P20=-1.9485873425573157, P21=0.5778412288679293, P22=3.905291870938857, P23=2.4755968220652544, P24=1.7707205988156882, P25=4.519436762750382, P26=0.924877585087553, P27=2.957551351380778, P28=1.7046796027205111, P29=2.8213675678653174, P30=2.437137052528815, P31=4.821730058378527, P32=3.686497478565638, P33=3.4356838499259372, P34=0.40121910042795067, P35=2.5016398590163145, P36=0.8851082177975426, P37=2.726402598236953, P38=2.066033478696479, P39=1.773209864402386, P40=0.768379884035619, P41=0.7249533626254705, P42=0.6960736035956808, P43=0.6729049484728589, P44=0.6731106711182291, P45=0.7318739720277028, P46=0.7110239214286483, P47=0.6662148787073662, P48=0.6592906009253376, P49=0.7237774016447412, P50=0.732244125970817, P51=0.7574539148799744, P52=0.0009185085458894731, P53=0.0001713613144384236, P54=2.698652039106463e-05, P55=4.8979467464747026e-05, P56=5.920479406034709e-05, P57=0.0004185524248569131, P58=0.002777123580499734, P59=0.008446322783154688, P60=0.01015293025314291, P61=0.01032803599228034, P62=0.02416755000901297, P63=0.027349777724052053, P64=0.007191719987692348, P65=0.0021221395328878865, P66=0.00625431556190115, P67=0.012698163236406138, P68=0.014529619661318686, P69=0.012583265780965233, P70=0.008996666608234694, P71=0.003800565591979783, P72=0.004017822426645995, P73=0.013532037551960484, P74=0.014025845545389198, P75=0.019830993798508563, P76=0.022633322956470962, P77=0.01590653761636075, P78=0.02143977582792712, P79=0.020168902367746357, P80=0.007226900844861943, P81=0.007323895794005525, P82=0.010760399080134063, P83=0.011470140342173672, P84=0.012922448252683969, P85=0.013313480613801838, P86=0.010302644861000321, P87=0.009090824368294618, P88=0.006917085092723168, P89=0.005070064307782036, P90=0.007389206552861629, P91=0.008500679433760941, P92=0.026134796089099327, P93=0.017786424358336113, P94=0.005192036716661611, P95=0.0013696498017597856, P96=0.0011849496397338644, P97=0.0029729180699659104, P98=0.013416539499891434, P99=0.032923078033027464, P100=0.010371717878139258, P101=0.002930861124252066, P102=0.0005482966557301422, P103=0.0005235477516650777, P104=0.001101727656415819, P105=0.0018602391315333802, P106=0.0019324787662502521, P107=0.0016694742165858205, P108=0.0008052733969813984, P109=0.00029090320578949733, P110=0.0013100037584538272, P111=0.004081374654303036, P112=0.0011182437894744438, P113=0.00013392713427285677, P114=0.0008509013031657346, P115=0.0008670993631817538, P116=0.0003220248358265062, P117=0.0018248585255360077, P118=0.0011404797949754572, P119=0.000514364699116489, P120=0.0010052933392633771, P121=0.0008645709943052703, P122=0.0008049768274937613, P123=0.0009667401627550513, P124=0.0009846983275474896, P125=0.0006076245145658953, P126=0.0002746472241514378, P127=0.0003112314579192717, P128=0.00017004641122810005, P129=0.00014662673545306874, P130=0.0005053192870481466, P131=0.00039282040819809674, P132=0.00026331394725783656, P133=0.0002995875136439732, P134=0.00019059409315134966, P135=8.980544678880754e-05, P136=8.587973253677489e-05, P137=4.257703014146143e-05, P138=3.1145296925631e-05, P139=3.400144453406566e-05, P140=2.6863277839923584e-05, P141=4.1128566021878886e-05, P142=5.041498307162458e-05, P143=7.148239116088227e-05, P144=7.692697974479742e-05, P145=7.08143700537185e-05, P146=6.99926246907587e-05, P147=3.330429723549749e-05, P148=1.5848626723611062e-05, P149=8.59904011391494e-06, P150=5.661072629320526e-06, P151=7.931746408123523e-06, P152=1.1015711507237163e-05, P153=1.5767156424392905e-05, P154=1.723529232322696e-05, P155=1.876733941374067e-05, P156=2.2277253103645742e-05, P157=2.855060672881288e-05, P158=1.5722227000220056e-05, P159=1.853876402473968e-05, P160=2.180041614004261e-05, P161=1.7984712684623887e-05, P162=1.1064354045281268e-05, P163=1.0640792020430824e-05, P164=1.384187782853061e-05, P165=1.939088310921021e-05, P166=8.339300786326721e-06, P167=5.678744827527098e-06, P168=6.054382299812375e-06, P169=5.0865618735291565e-06, P170=5.828325129608553e-06, P171=5.778999371645324e-06, P172=2.873869430588086e-06, P173=3.1868884081834375e-06, P174=4.187510591506208e-06, P175=6.292584155769017e-06, P176=7.565521186763235e-06, P177=6.3790772209336e-06, P178=2.002381806871908e-06, P179=2.4799944230679066e-07),Row( P0=-556.8989665248348, P1=50.704407965162886, P2=-18.55202111548884, P3=-3.964472564630579, P4=-14.631064533932614, P5=-14.658534625255006, P6=-13.188842016334226, P7=0.8065509554060492, P8=-11.06610823493217, P9=-4.948562616155394, P10=-8.493144202816229, P11=-3.509008225700854, P12=0.09331350344589887, P13=-4.648606980187983, P14=3.495635461312364, P15=1.1421935812587354, P16=4.970293235833209, P17=9.04749623405702, P18=4.099887113992083, P19=9.532947247916477, P20=1.5643335825562763, P21=3.154742449475406, P22=6.1327754829433925, P23=4.948945902181284, P24=2.0510307648034565, P25=3.845151368952212, P26=-1.4052525888339793, P27=0.21757584630958604, P28=0.8157194387425982, P29=1.3372764627227773, P30=1.0948019373962992, P31=3.5316097997673572, P32=0.5437421780489381, P33=2.9249985663992644, P34=0.9684230416806933, P35=2.26464880041113, P36=1.3839811051363107, P37=3.6946376843278266, P38=1.4662106989593984, P39=0.46376461693992316, P40=0.698543268404327, P41=0.6429095369720196, P42=0.6148298821764013, P43=0.5860433595469079, P44=0.5994015265731978, P45=0.6331236470765106, P46=0.6873080224871938, P47=0.6515873784322893, P48=0.5999113959817317, P49=0.6200357169108586, P50=0.6696574535716936, P51=0.695290848679513, P52=0.002734546574241173, P53=0.00031845407188553856, P54=2.7835246937189738e-05, P55=5.167073703126541e-05, P56=7.926768370481908e-05, P57=0.00038984405204229813, P58=0.0015501029947170327, P59=0.0018556679664739845, P60=0.001282688610461566, P61=0.004884152167401755, P62=0.0701740545484942, P63=0.1279301587678763, P64=0.055585877873941016, P65=0.043698373188271536, P66=0.08136688709020767, P67=0.12158333238371952, P68=0.08804612842005209, P69=0.05070711643197777, P70=0.038608399625209074, P71=0.029720572640339917, P72=0.040642329190481996, P73=0.07006243264423215, P74=0.0757444755163509, P75=0.08000633854875316, P76=0.07774561805503476, P77=0.1380277115060648, P78=0.2379183925438086, P79=0.2011464252724936, P80=0.12317913067974796, P81=0.13212162664553995, P82=0.16382407758518616, P83=0.19137268011009362, P84=0.25603238088550834, P85=0.23570126889569337, P86=0.20730242499072987, P87=0.12585275929748194, P88=0.05638426035925649, P89=0.041665988687985114, P90=0.12062792537343327, P91=0.23179769543832648, P92=0.20610332473691906, P93=0.29669696005002183, P94=0.5429262718523735, P95=0.37450092224280396, P96=0.16720531743795777, P97=0.09534846044194743, P98=0.04507707010175129, P99=0.03602213383325242, P100=0.08657513104599326, P101=0.08577395873275549, P102=0.03268366015061057, P103=0.022638047207169335, P104=0.061778262593364226, P105=0.08430676933333096, P106=0.04778845647269721, P107=0.036720241393754606, P108=0.025479905403268895, P109=0.038150423827128625, P110=0.11201786968016218, P111=0.1551246313045995, P112=0.04786745650656977, P113=0.005903338720129745, P114=0.011363597610497198, P115=0.01847656158787548, P116=0.012616098248159034, P117=0.005566299565963344, P118=0.003864905438792953, P119=0.008891778414325466, P120=0.012701359469110919, P121=0.004783217271788709, P122=0.00722689232319019, P123=0.008647898282503013, P124=0.006397271625002724, P125=0.00437513878074689, P126=0.004298633201120012, P127=0.003618532857489251, P128=0.00661861949768763, P129=0.007219396116210785, P130=0.004881301664941118, P131=0.0027725194414461233, P132=0.004241718373998992, P133=0.004521658911808662, P134=0.0022088453287066293, P135=0.0024597908799992674, P136=0.0021752611050405253, P137=0.00046099047593783774, P138=0.0005267444843880098, P139=0.00045228573679617607, P140=0.0005556381468759525, P141=0.0007634799071476278, P142=0.0009924437838214552, P143=0.0009132096268835516, P144=0.0007787139536262159, P145=0.0008938886118611053, P146=0.0011137730590238732, P147=0.0005969495904849713, P148=0.00022105227316816497, P149=8.032258844533232e-05, P150=3.903233952386632e-05, P151=4.771293938178312e-05, P152=8.039066729268954e-05, P153=0.00011246036998752811, P154=0.000184343134160823, P155=0.0002153334428642379, P156=0.0002927083635943189, P157=0.0004055590275867722, P158=0.0004928947991558447, P159=0.0006335779264969902, P160=0.0007119108541171245, P161=0.0004878515573708475, P162=0.0002453375179946063, P163=0.00014995505712581117, P164=0.0001238516929823466, P165=0.00012081853481586676, P166=8.352123804718034e-05, P167=6.346941119096967e-05, P168=4.867637972700517e-05, P169=3.80829499033148e-05, P170=3.713356504339999e-05, P171=3.198034344822919e-05, P172=3.9725080478738694e-05, P173=8.27300839259991e-05, P174=7.866638186952983e-05, P175=8.807790066257187e-05, P176=5.713592027876074e-05, P177=3.3549647776107074e-05, P178=1.3064593794118287e-05, P179=1.2824498758698524e-06),Row( P0=-747.7910431294534, P1=78.7857686376848, P2=-4.069759497157757, P3=9.344369167939858, P4=-1.7795007509808924, P5=-5.8336299198116865, P6=-1.5431984722171594, P7=3.0943019039972657, P8=2.1351913130191296, P9=0.354354922134012, P10=-3.574517578603481, P11=-1.1944938221650017, P12=-2.145754203656173, P13=-0.7319207964448006, P14=5.981125010010449, P15=-3.9567888583197837, P16=0.03332454297522714, P17=3.8412861782096224, P18=-2.9773452431084775, P19=3.255445123371578, P20=-3.0886180045876417, P21=-2.0327334827273247, P22=0.578292419570319, P23=-2.321646437973656, P24=-0.06929653386157507, P25=0.18286246825951377, P26=2.2577470072410395, P27=3.2845919934959, P28=2.8749671365377103, P29=3.9762734371007236, P30=0.31317729169525305, P31=2.6852573495801386, P32=0.6929982087946926, P33=1.545235660349272, P34=1.62108376167602, P35=1.7574119983313863, P36=1.3701425594929888, P37=2.614089566159055, P38=2.8599566721827974, P39=2.6991035714997698, P40=0.7326678933605628, P41=0.7745037323269951, P42=0.7897531832585952, P43=0.7779445432419152, P44=0.7480353560192833, P45=0.6881601775943942, P46=0.6547738867283194, P47=0.6630966023080063, P48=0.7255609293035303, P49=0.7451681183389304, P50=0.7400325330965092, P51=0.7380078202387748, P52=2.825072168297486e-05, P53=1.4167302359141539e-05, P54=2.914983161262032e-05, P55=7.432032956379918e-05, P56=0.0001082213074371405, P57=0.001005817987855355, P58=0.0023429784346339143, P59=0.002214660441466604, P60=0.003875561732367416, P61=0.0035598369775779203, P62=0.0016371638486135664, P63=0.0006835781114452541, P64=0.0009165092382821137, P65=0.003481665776574442, P66=0.002179396618500221, P67=0.0013091900898172136, P68=0.0018804862249523543, P69=0.0041380684574603074, P70=0.006221127430892427, P71=0.003432606225604681, P72=0.002647327709848713, P73=0.0016564633583538, P74=0.0007706369715346889, P75=0.0013231712412015313, P76=0.0025924799808080567, P77=0.0035251616256151046, P78=0.006111663045437768, P79=0.008427481584963379, P80=0.005032952000918543, P81=0.005599578479190557, P82=0.005639145699700624, P83=0.003755521657977688, P84=0.006727694654890111, P85=0.011727151375453422, P86=0.0064482196085592575, P87=0.004632982846671811, P88=0.011340064178930801, P89=0.006342586399772316, P90=0.0033397657780053166, P91=0.005756241642942744, P92=0.005806371817711367, P93=0.00392533846272565, P94=0.0069306253881451976, P95=0.004154450688576974, P96=0.002852893852905214, P97=0.0030100546538260136, P98=0.0028510815143792884, P99=0.0022235256369406626, P100=0.0013536382769964558, P101=0.0006861968443503125, P102=0.00015759353176596634, P103=0.00029240238855711236, P104=0.0005206642410164324, P105=0.0006353886336064573, P106=0.0007571440734438698, P107=0.0011549516629048577, P108=0.0008815843322825217, P109=0.0006459811425061985, P110=0.0011091671344336225, P111=0.0015124876407718024, P112=0.0008223329828343123, P113=0.00036998349176516803, P114=0.00032271618972190525, P115=0.00027144831846415975, P116=9.782250459222492e-05, P117=0.0003244202449265257, P118=0.00021711300436880857, P119=0.00014150333287455734, P120=0.00010958972183901434, P121=7.321832086592805e-05, P122=6.331807547187774e-05, P123=0.00012394986585813027, P124=8.316305942448748e-05, P125=6.32543608994406e-05, P126=9.324774109740126e-05, P127=8.268011271835742e-05, P128=5.9958226867208726e-05, P129=5.3885123276670315e-05, P130=8.301252667943686e-05, P131=5.153713586099156e-05, P132=2.739634647590448e-05, P133=2.1301793599001518e-05, P134=1.4634948805452912e-05, P135=9.132132487706427e-06, P136=7.863746769349702e-06, P137=2.369936216950033e-06, P138=3.5927578527178144e-06, P139=4.798036557578957e-06, P140=7.296813032756311e-06, P141=8.48432695970948e-06, P142=8.001324503569367e-06, P143=1.3717023714137476e-05, P144=2.911065795397767e-05, P145=3.492036203067028e-05, P146=1.860543920582511e-05, P147=4.044234337063077e-06, P148=3.2477926569476415e-06, P149=2.297176175562654e-06, P150=1.5627057152296425e-06, P151=1.0973920889366333e-06, P152=6.634988488293414e-07, P153=1.0250659227830254e-06, P154=1.3388336661603742e-06, P155=8.775575881252195e-07, P156=3.8296190943093455e-06, P157=4.459685290521609e-06, P158=4.434283457217099e-06, P159=3.7711389503931417e-06, P160=1.6716972974827042e-06, P161=1.2266351602318648e-06, P162=7.027899330644269e-07, P163=4.2573485302030424e-07, P164=2.6941374959205964e-07, P165=5.875754095293055e-07, P166=5.994102356097914e-07, P167=5.311271303508212e-07, P168=3.926703864850141e-07, P169=3.437733996885039e-07, P170=1.6824630358943456e-07, P171=1.9565734337328722e-07, P172=1.8752816896036485e-07, P173=1.6429507558276818e-07, P174=2.5445007868079566e-07, P175=2.9904805680329863e-07, P176=3.107073810923453e-07, P177=4.689446856535508e-07, P178=2.2935993251596827e-07, P179=2.6588946824541936e-08),Row( P0=-712.9938975712657, P1=58.451858583068436, P2=6.267311441912715, P3=8.088929674963088, P4=-0.7330268739155988, P5=-1.1042609002861414, P6=1.15957654135807, P7=5.808559864841517, P8=-0.02952201158897375, P9=2.0962706926478325, P10=0.8430147736114886, P11=1.5077004263018785, P12=0.335874730764292, P13=-2.767171477675084, P14=0.9581640986089162, P15=-0.3853057179788535, P16=1.4501145435047382, P17=2.057631622826746, P18=0.8180764294323263, P19=4.472570678109407, P20=-0.17662417556806656, P21=2.0171640694506343, P22=3.103360700311938, P23=2.802615977237915, P24=0.9896193961983217, P25=3.520249352846959, P26=2.089716285545456, P27=4.207935533416056, P28=3.9986449579236996, P29=4.280729673150663, P30=0.7059251602184222, P31=3.292092591257227, P32=1.1570196230817207, P33=3.533984063082172, P34=1.711555051805388, P35=2.4811803956531957, P36=0.41823312848068467, P37=1.1894421433634506, P38=2.0154523007901153, P39=1.7660495858794918, P40=0.7695351083952049, P41=0.7442783841400381, P42=0.7308662362164545, P43=0.7313588549188812, P44=0.7494652249157664, P45=0.7335139989141444, P46=0.7357042015485344, P47=0.7507393038647835, P48=0.7735781314291095, P49=0.7582569055833215, P50=0.7463712181280705, P51=0.7726174010567225, P52=0.0013538828576914346, P53=0.00018920757507776643, P54=3.798841571160397e-05, P55=6.118640856929232e-05, P56=0.00016559951681554847, P57=0.0014211862299816124, P58=0.0033480981825498145, P59=0.009436639995917425, P60=0.012145260513626563, P61=0.006793671212530821, P62=0.0068062059124138615, P63=0.013381796940145253, P64=0.007360440669403735, P65=0.0022282691057107324, P66=0.003293187755993209, P67=0.007051848340273143, P68=0.009740766707285287, P69=0.011603369725604043, P70=0.010258936867033983, P71=0.0041311850685190245, P72=0.0013039142502728768, P73=0.0011836939325215376, P74=0.0019304306425703428, P75=0.003853888380957361, P76=0.0062684108019135315, P77=0.014940619984386444, P78=0.015760430568578888, P79=0.01484307765871052, P80=0.01041476082029514, P81=0.01627998780629213, P82=0.01965857893518799, P83=0.006008024197694517, P84=0.0030324176183326927, P85=0.0052426260673013704, P86=0.0033927827228193343, P87=0.0030098169432605077, P88=0.003071959609962434, P89=0.0027431358343288, P90=0.0037061944585259515, P91=0.0022739747628542618, P92=0.0023721276669846883, P93=0.0017377936638327152, P94=0.0011124784460999888, P95=0.0009752579640063343, P96=0.003379891631433383, P97=0.01073973122691605, P98=0.005605234232953225, P99=0.002601823599076977, P100=0.0018493546503466417, P101=0.0012222722888957415, P102=0.001581507537499802, P103=0.0014273056472615532, P104=0.0004987434862473619, P105=0.0011911224205093077, P106=0.0011439114512868766, P107=0.0012777818205936947, P108=0.00042277800232771594, P109=3.5561015289071116e-05, P110=0.00011102058049761579, P111=0.0001477086178488606, P112=0.00010278527603707903, P113=7.395831880581009e-05, P114=5.302456306240957e-05, P115=4.029653688715598e-05, P116=7.509442260641866e-05, P117=0.00012121069957338558, P118=0.00013932973746012996, P119=5.6922708986196366e-05, P120=0.00011409480840724922, P121=0.00012803369177077442, P122=0.00010063308893460074, P123=9.632078833480779e-05, P124=7.445147804754249e-05, P125=5.569559939256262e-05, P126=4.5075597994686366e-05, P127=5.6763351538565644e-05, P128=5.5595515083993184e-05, P129=4.4890737226462724e-05, P130=5.311704345174911e-05, P131=5.706978860418741e-05, P132=5.231411635238394e-05, P133=8.67484780317074e-05, P134=9.154884072602428e-05, P135=2.8466149271181256e-05, P136=3.2640840608873775e-05, P137=3.9813557395261796e-05, P138=3.330928332222404e-05, P139=3.878597014949885e-05, P140=3.2652603165729966e-05, P141=3.0787034200323605e-05, P142=4.0611171797371205e-05, P143=5.910420618357384e-05, P144=5.479053087511337e-05, P145=3.8201441814156206e-05, P146=3.305279980081261e-05, P147=1.672808472065325e-05, P148=1.7728550841277418e-05, P149=1.4577796803968361e-05, P150=1.7243137821632485e-05, P151=9.491395738759018e-06, P152=1.2710491613388912e-05, P153=1.4606292836409007e-05, P154=1.8868707667049387e-05, P155=2.8826123725379763e-05, P156=1.9466492900030978e-05, P157=2.159323149677323e-05, P158=3.760256915293049e-05, P159=2.9189510931001695e-05, P160=2.829658114664788e-05, P161=1.4772398421115607e-05, P162=6.177070715023318e-06, P163=7.972075923046812e-06, P164=1.3576321073536949e-05, P165=9.24826422621904e-06, P166=8.13939216887795e-06, P167=5.777636942462524e-06, P168=4.233388774943003e-06, P169=4.340768903188756e-06, P170=3.6486363399730794e-06, P171=2.7930869659558644e-06, P172=1.6177858139946213e-06, P173=2.6749419611858116e-06, P174=9.10169214350312e-06, P175=9.530252439638168e-06, P176=4.362127188451896e-06, P177=4.932512894295932e-06, P178=1.8599749350871834e-06, P179=5.349901308130754e-07),Row( P0=-749.8444360911391, P1=72.9026763217388, P2=-2.3011795096505194, P3=10.817226133229664, P4=-2.1748980620843703, P5=-2.9969459709120247, P6=1.9233135777367043, P7=4.205350604914653, P8=4.310247741431771, P9=0.2119924999950896, P10=-3.6438145252586747, P11=0.5242397285587951, P12=-1.9673520193470537, P13=-0.2820170641109674, P14=2.6786632438567026, P15=-2.9095186516761498, P16=2.0358784375382255, P17=1.4556554209306285, P18=-1.2630691332065236, P19=3.35376808958917, P20=-3.920428509284835, P21=1.6284068058307484, P22=0.6129516890642349, P23=-0.4088567632087615, P24=-2.2226275244222573, P25=-1.4877087346697198, P26=0.3844468519399686, P27=1.1396871736875982, P28=0.6010884507608059, P29=2.7696478256739354, P30=0.9814641706263889, P31=2.699601239777185, P32=1.5012423914647608, P33=0.6096719450080412, P34=0.6614014950968119, P35=0.3821039802097048, P36=0.8331066697716984, P37=2.845381363068765, P38=3.8046045962656296, P39=4.619503677275993, P40=0.7797989359717403, P41=0.7490363227430309, P42=0.7723932691418071, P43=0.7813003100499519, P44=0.7431350489142521, P45=0.7419525295478498, P46=0.717847951664404, P47=0.7298244102131451, P48=0.7913933143601997, P49=0.7704570137811654, P50=0.7286195837630681, P51=0.7535418600762914, P52=4.278334725397584e-05, P53=1.3296020678859862e-05, P54=1.659983824198962e-05, P55=4.7439315595162584e-05, P56=0.000299658558149406, P57=0.004193013435162242, P58=0.004883346622816401, P59=0.002464242912031512, P60=0.003100913183853621, P61=0.0019479879433852423, P62=0.0007493851523548321, P63=0.0017597829078083414, P64=0.005204186174847693, P65=0.003987522809340035, P66=0.0015474393692279877, P67=0.001832899884315797, P68=0.0015993273505649922, P69=0.0022892534439206396, P70=0.004887615426913529, P71=0.004049533588468317, P72=0.0013659221620156477, P73=0.0006473080387010312, P74=0.0004634710615023347, P75=0.0008681638840695911, P76=0.0012738128423732172, P77=0.0032573372705915695, P78=0.00544485385882658, P79=0.010365850346940172, P80=0.00978148881948595, P81=0.0029292001293486193, P82=0.001231761749322634, P83=0.004467239615148497, P84=0.006865915906991944, P85=0.005743528620046074, P86=0.005792685511184728, P87=0.0025636669335826184, P88=0.003870007554617718, P89=0.00905463744330356, P90=0.003790025603575308, P91=0.003052331242965846, P92=0.005314756517361104, P93=0.004851904492496689, P94=0.004367196292720588, P95=0.0005884320801605937, P96=0.0016745788534820987, P97=0.0072272136585434, P98=0.005274181020908853, P99=0.005988764376787225, P100=0.0011528342283350778, P101=0.0003948330366480322, P102=0.000328813537434293, P103=0.00034321832767057075, P104=0.00038701472505682304, P105=0.0006895293901606664, P106=0.0013841389501321206, P107=0.0004805686363896065, P108=0.0003047730224052733, P109=0.0008111485448956958, P110=0.000629117728943465, P111=0.0005617737941782306, P112=0.00044757081703782574, P113=0.00014936696818331716, P114=6.486939952227971e-05, P115=0.00014764608717185088, P116=6.536309730188339e-05, P117=0.00022870925869800294, P118=0.00011808728478165274, P119=6.341315462470441e-05, P120=9.475121798617923e-05, P121=8.826192221180997e-05, P122=0.00010915357454466991, P123=0.00011245291197055409, P124=5.5095599740328736e-05, P125=6.1878618476654e-05, P126=5.568961362371053e-05, P127=5.944176177711259e-05, P128=5.992317013872161e-05, P129=0.00010397982868773788, P130=6.698342592812683e-05, P131=6.745540163256439e-05, P132=4.917042020453449e-05, P133=4.1800851705075945e-05, P134=2.8209987287637227e-05, P135=1.634034744497268e-05, P136=1.9283540127940524e-05, P137=5.439483184358812e-06, P138=7.721132187161016e-06, P139=7.132596279797123e-06, P140=1.2511939184009339e-05, P141=1.2308521537263932e-05, P142=9.495856608522074e-06, P143=1.191158168101966e-05, P144=2.5633435517154987e-05, P145=8.576577712576865e-05, P146=5.645163128475056e-05, P147=1.8835662445642413e-05, P148=7.427235314068095e-06, P149=4.5928485309240816e-06, P150=2.5650977119090333e-06, P151=2.748206733753893e-06, P152=1.1902269684671531e-06, P153=1.2844338062976774e-06, P154=1.6897327102738434e-06, P155=2.1472005323818e-06, P156=5.0220886287045955e-06, P157=2.945209165243713e-06, P158=2.84654612646054e-06, P159=3.6054275781303246e-06, P160=5.589655617792387e-06, P161=5.278670928908527e-06, P162=2.454761332052239e-06, P163=1.174463284036296e-06, P164=6.693440639160705e-07, P165=6.708756691110389e-07, P166=1.0821944430629225e-06, P167=1.0742919716059517e-06, P168=8.489492657863833e-07, P169=5.035090134831193e-07, P170=4.559709502602021e-07, P171=4.351009948850102e-07, P172=2.4807494517746893e-07, P173=2.2450478019644548e-07, P174=4.5351171666894117e-07, P175=3.804008610300931e-07, P176=3.7798310940506905e-07, P177=6.243226559093233e-07, P178=6.118143382006454e-07, P179=1.0199506698279954e-07),Row( P0=-608.1681968932742, P1=54.144476197827444, P2=-16.068388185828308, P3=7.616592183997082, P4=-7.683465213971743, P5=-12.437360370177153, P6=-4.871252541735011, P7=4.04292757747548, P8=-3.3833632795510833, P9=-1.2256933424229233, P10=-6.434513091287693, P11=0.43256560738217836, P12=-0.3744319383753839, P13=-4.9642551301049345, P14=5.54658953297687, P15=-2.662223506055052, P16=1.6281168325001583, P17=3.9251226570764985, P18=-2.768355001684507, P19=6.184909426069529, P20=-0.7184681244214506, P21=1.6112634248179538, P22=5.445249725965691, P23=1.2995258215880026, P24=2.554759598682394, P25=3.716195445421978, P26=0.38682077309075136, P27=5.012286842326815, P28=4.200993401420969, P29=3.7847310968927954, P30=5.047347255916907, P31=5.9612423013683395, P32=5.2266212231811595, P33=3.501963755695198, P34=1.4059648199499926, P35=1.556902515266957, P36=-0.47535580060166555, P37=1.0213489498701782, P38=3.3214897647138466, P39=1.9313244632907594, P40=0.7469256118932994, P41=0.7349082177162062, P42=0.6540789129505225, P43=0.6686901635091428, P44=0.6880614794546682, P45=0.7118764000588569, P46=0.676739658966472, P47=0.6913143482220722, P48=0.7194742984936192, P49=0.6671724299753353, P50=0.6645279330047161, P51=0.7296319617040001, P52=0.023186527621559896, P53=0.0035574756943826426, P54=0.0001525550947086013, P55=0.00011996080283357693, P56=0.00020989467892135836, P57=0.0005389114315242396, P58=0.0018371027115884216, P59=0.005475299705676239, P60=0.009296838776011234, P61=0.019837380060424577, P62=0.040698724330672245, P63=0.05081023260588573, P64=0.027363099928403727, P65=0.008977677685068313, P66=0.00934058978050516, P67=0.015721945909071864, P68=0.020985376749346026, P69=0.01606932579176277, P70=0.009975922269172453, P71=0.006569924894053496, P72=0.013747413413900188, P73=0.023596339851385647, P74=0.030119909944979213, P75=0.04301135100789215, P76=0.06529788217174719, P77=0.19259208050776283, P78=0.12785391519006922, P79=0.028150131445858977, P80=0.007960200187990197, P81=0.006573759556539275, P82=0.008208498726075014, P83=0.011118778184558607, P84=0.016421663085879046, P85=0.047644783131110785, P86=0.05483817771096083, P87=0.021326562696270295, P88=0.021166585525525188, P89=0.016699946867610324, P90=0.013196284013925799, P91=0.015883143303111174, P92=0.06831950611409873, P93=0.028875982159909514, P94=0.019283707967050295, P95=0.02084984506474861, P96=0.01648941463116192, P97=0.01948281616065355, P98=0.04376311130379962, P99=0.05557650766928201, P100=0.00837067667555555, P101=0.0038815506147461608, P102=0.003465807995295744, P103=0.005394509904816229, P104=0.0033903184803381627, P105=0.0017240274467094122, P106=0.0023356619725205304, P107=0.00721194347831741, P108=0.0046589411486905345, P109=0.0028847647332231317, P110=0.005259537538405934, P111=0.008538959842372181, P112=0.005897316120466012, P113=0.0037828648354051504, P114=0.002082774719544591, P115=0.0010832043648355172, P116=0.002085743148766508, P117=0.0035551718468037155, P118=0.0027523797886748456, P119=0.0015736870382601525, P120=0.0029419863443044176, P121=0.002700633865136515, P122=0.0035840564551244637, P123=0.002715962536457003, P124=0.0020527636292146855, P125=0.0020443475555487803, P126=0.00815499662960462, P127=0.0027693515267955865, P128=0.0005819629534519661, P129=0.0005352887667572226, P130=0.0006536790234976552, P131=0.00044175912373236497, P132=0.0010618567217924914, P133=0.0007446038157921825, P134=0.0011209109883956452, P135=0.000551467286613585, P136=0.00031198062357594447, P137=0.00014533968271110024, P138=0.00020490976316380128, P139=0.0002187763553816249, P140=0.00021158026792407957, P141=0.00033036703646755074, P142=0.0002476500768587094, P143=0.0004869900938847695, P144=0.0006992920229260863, P145=0.0005993499529196408, P146=0.0003721411965085551, P147=0.00012664479278426085, P148=4.77183901114953e-05, P149=3.219264879562125e-05, P150=2.1599484357367925e-05, P151=2.147926084836447e-05, P152=3.1696529732604315e-05, P153=0.00013636530036314546, P154=0.0001660551011633253, P155=0.00020854722337800872, P156=0.00021035355710825187, P157=0.00041655131032857726, P158=0.00031312270657603796, P159=0.000257899006822589, P160=0.00019153630609360916, P161=9.916445259213467e-05, P162=0.00010349281849692564, P163=0.000131774321941577, P164=0.00010711820241318167, P165=0.00011283381408141026, P166=6.908158773778449e-05, P167=6.11528317914335e-05, P168=2.5280956219036093e-05, P169=2.3030499447744997e-05, P170=2.1324262343805216e-05, P171=2.685200904312016e-05, P172=1.5078206376208236e-05, P173=1.585768888004266e-05, P174=2.699281360654481e-05, P175=4.785071740556307e-05, P176=8.785767909518325e-05, P177=6.466614318631936e-05, P178=2.670066645811989e-05, P179=2.181837181485148e-06),Row( P0=-652.0940022162001, P1=57.65308352780513, P2=-1.8955152512826472, P3=2.0200766113005018, P4=-7.1378380494087015, P5=-5.839763317588189, P6=-0.00039972064155239156, P7=2.337528081083735, P8=-2.6720630794600493, P9=-1.4471461008484332, P10=-3.178713604849114, P11=-1.5908663541297252, P12=-2.6766630208785958, P13=-4.4819594490597865, P14=0.038439918808657755, P15=-1.3203226479852326, P16=2.906995049241987, P17=2.8318484596071962, P18=1.1174539515697648, P19=4.1693619220029055, P20=-3.5534854994956913, P21=1.866002025259068, P22=0.3645102286845466, P23=-0.03618401006831619, P24=-1.1638834137383451, P25=1.5546443146345954, P26=0.9656192539404583, P27=0.9248299526141721, P28=1.5486335479244706, P29=2.766790049075751, P30=-0.3601131218733894, P31=4.32704599940148, P32=0.5797125486535606, P33=3.0410733886131576, P34=2.085870741652001, P35=3.107408384983873, P36=1.1441759279519814, P37=2.537784105498476, P38=1.454781977144998, P39=2.0297290329675586, P40=0.7657927170773926, P41=0.7300356995207696, P42=0.7174428006865553, P43=0.7151740996924427, P44=0.7132389908042273, P45=0.7343143912662733, P46=0.704634202586718, P47=0.7051529984339934, P48=0.7351660334117144, P49=0.7628904911264142, P50=0.8159289756639035, P51=0.7959893288723427, P52=0.00018507989919128666, P53=3.4510336999821386e-05, P54=2.707477361390997e-05, P55=6.550497393928335e-05, P56=0.00032105601140509704, P57=0.0024136715721235873, P58=0.005017383114975441, P59=0.008438090390061083, P60=0.011273806920114107, P61=0.01472175125914363, P62=0.009306613516595607, P63=0.0042473854835500206, P64=0.003960852903030971, P65=0.010879761404039, P66=0.019681316954021914, P67=0.020240560739117862, P68=0.016796491319716767, P69=0.011710938477648447, P70=0.01605851100829352, P71=0.0160812841901402, P72=0.01286598615643554, P73=0.006966772549540195, P74=0.0031234571957528276, P75=0.0019833602008732833, P76=0.010458282216799024, P77=0.028048731675499303, P78=0.028535332266419165, P79=0.037781279587019385, P80=0.0456212368364336, P81=0.048649709208488205, P82=0.08496529359441088, P83=0.033335684955223724, P84=0.014911766758235403, P85=0.02811393130787378, P86=0.01670727654832145, P87=0.01420486487160278, P88=0.012031607920619403, P89=0.00788206166373146, P90=0.011076738119738867, P91=0.01320307214852878, P92=0.021098534230485824, P93=0.014219194842093245, P94=0.010137861955755672, P95=0.005606637504593133, P96=0.009881597320722224, P97=0.018562201249206496, P98=0.01619115134694161, P99=0.006527122918881644, P100=0.01097278080490309, P101=0.01654476391480002, P102=0.006601593030416598, P103=0.004724231379431633, P104=0.0014012990136237945, P105=0.0014258824889717202, P106=0.002510723861187748, P107=0.0012717806279678465, P108=0.00042295395432288856, P109=0.0005310603272675532, P110=0.0008233079797023283, P111=0.0008939211827502806, P112=0.0006675683153582639, P113=0.00017562628389813202, P114=0.00015710017254942722, P115=0.0002754439830253699, P116=0.00014850140129824766, P117=0.00026607527155085234, P118=0.0002688174441565283, P119=0.00015914362976520032, P120=0.0002809826450643877, P121=0.0002881664427335365, P122=0.000379327721126842, P123=0.0003944080077021744, P124=0.00018102701758942314, P125=0.00023665088553103956, P126=0.00019321867046666157, P127=0.00012147206340532971, P128=0.00011719832012323007, P129=0.00018495757609930677, P130=0.00012902258628961209, P131=0.00012216669821601189, P132=0.0002085790933455417, P133=0.00023963399907788143, P134=0.00016956695324242825, P135=0.00012133481747291061, P136=0.00011553997664145985, P137=7.273729450882446e-05, P138=0.0001324729573139075, P139=7.007908224050267e-05, P140=0.00011283449553017355, P141=7.325159739025725e-05, P142=5.8409390909106384e-05, P143=6.651385164090219e-05, P144=6.403156615045454e-05, P145=0.00011981363664816893, P146=6.36271308195821e-05, P147=6.131991242243142e-05, P148=3.470012284871929e-05, P149=6.163837931020748e-05, P150=9.260534224588983e-05, P151=5.6909878655880876e-05, P152=4.2343308151281325e-05, P153=3.0836501962334675e-05, P154=1.9990028266130613e-05, P155=1.433164900415653e-05, P156=2.3026935421486434e-05, P157=4.0072144115278134e-05, P158=4.7898237807261514e-05, P159=7.339739730438621e-05, P160=8.753274312245437e-05, P161=0.00012010134907461345, P162=5.951634916572541e-05, P163=5.1730379927644617e-05, P164=4.219296888958001e-05, P165=2.1856436266987355e-05, P166=2.405851996591568e-05, P167=2.1542668241329653e-05, P168=1.5197403837872e-05, P169=1.1397655230925894e-05, P170=7.202112982081675e-06, P171=5.914735243960558e-06, P172=1.1376844403878033e-05, P173=1.7218495949640124e-05, P174=2.4063257036732075e-05, P175=2.4032291271325154e-05, P176=2.6055962115004396e-05, P177=2.1778418475595797e-05, P178=9.227783105689051e-06, P179=8.332154903505552e-07),Row( P0=-727.5227331455371, P1=75.38816353295192, P2=3.603829658574558, P3=2.168675906912757, P4=-2.4542674114386402, P5=-2.189431528971711, P6=-0.5052948627458023, P7=3.148216621832778, P8=3.8568199194974477, P9=0.5514527265424536, P10=-2.5678090486939253, P11=-1.641302439301929, P12=-1.9614112775245247, P13=-1.9697664769745957, P14=1.2266925348207378, P15=-3.2388645727719187, P16=1.0013645184489872, P17=1.7605092529849764, P18=-0.6652548045032209, P19=2.165676182513791, P20=-3.683102537914808, P21=0.5333831352999499, P22=-0.2829213502121395, P23=-2.1538975992191425, P24=-2.226351301523097, P25=-0.8291933888689947, P26=-0.6292001696861848, P27=0.23540714808913318, P28=-1.0911407060827119, P29=1.9466372603175384, P30=-0.8421316687529037, P31=2.6778543715934395, P32=0.8963991059147224, P33=1.5792239937041008, P34=1.560229192960825, P35=1.3710572489461228, P36=2.033701942945551, P37=3.8971402437960077, P38=2.761525787885113, P39=3.9915123448873038, P40=0.7628935261219204, P41=0.7248522866065299, P42=0.743858943467599, P43=0.7437736134118043, P44=0.7542976180854002, P45=0.7450653056528644, P46=0.712858702266018, P47=0.715331785922032, P48=0.7545082510805697, P49=0.7592402122204679, P50=0.759127468446048, P51=0.7638481014442352, P52=5.8398508244514466e-05, P53=3.2192607753578006e-05, P54=1.9276391924176774e-05, P55=8.728168031163151e-05, P56=0.0007553650290098248, P57=0.007569475709541834, P58=0.009393517384473013, P59=0.006165468720601855, P60=0.003988851995066672, P61=0.0014154003770667641, P62=0.0009448374957543102, P63=0.0014528596265808498, P64=0.0024246894769304464, P65=0.007344233731334999, P66=0.011481255728189166, P67=0.013238419105844237, P68=0.007023376823426818, P69=0.00377551694698044, P70=0.0024423671456185233, P71=0.0018489491938066796, P72=0.003460922890737203, P73=0.0025811082140721653, P74=0.00135303372014237, P75=0.0014420739976913003, P76=0.0018640576745426792, P77=0.005157438325247156, P78=0.009694375187079355, P79=0.017731511315577264, P80=0.011325187583929941, P81=0.0034461774365038326, P82=0.0027092522689833266, P83=0.003686922753754, P84=0.004550306951834694, P85=0.005017484562202849, P86=0.0043407399593174715, P87=0.003966914601667289, P88=0.005749253723128344, P89=0.003867495727189969, P90=0.005776436128034408, P91=0.007720677870696071, P92=0.013891861222413034, P93=0.00757313604854745, P94=0.005098667260071964, P95=0.0038449344847781555, P96=0.0034026505630707373, P97=0.003884543060624262, P98=0.0033412514426381687, P99=0.0015659976156620872, P100=0.0005337032914578641, P101=0.00041897306192728454, P102=0.0009277616830250585, P103=0.0012721993920706185, P104=0.00042871916888571307, P105=0.0005952214573130901, P106=0.0008490697073947451, P107=0.0009019946711717716, P108=0.000288694128558439, P109=0.00034617364201645695, P110=0.0009675407110003175, P111=0.0010409953504569131, P112=0.00027752643474390315, P113=0.00021446844959168533, P114=0.00016235844718360414, P115=0.00016378698455471723, P116=9.368508155213357e-05, P117=7.504310677722104e-05, P118=5.102671476971919e-05, P119=3.9358957698551856e-05, P120=0.00010135551707718492, P121=0.00011892355045449283, P122=0.00013170063549986687, P123=0.00010664660629224262, P124=0.00011771069764647181, P125=6.492387698544817e-05, P126=9.178927155792645e-05, P127=7.459831120991262e-05, P128=4.724824323210826e-05, P129=7.961786165642467e-05, P130=4.718706208021864e-05, P131=7.660274028094624e-05, P132=8.347929677587259e-05, P133=6.254493852598272e-05, P134=4.069652831322301e-05, P135=7.887571201685802e-05, P136=8.801395201125423e-05, P137=3.7108163112034016e-05, P138=1.7917338823971207e-05, P139=8.96646317527305e-06, P140=1.2529728343791049e-05, P141=1.1309373709347386e-05, P142=1.7145925405913692e-05, P143=2.9231790535572816e-05, P144=2.404432217597746e-05, P145=3.3923883030031996e-05, P146=2.9202878235404802e-05, P147=3.9011112503541886e-05, P148=2.7606602281281945e-05, P149=1.4231995514754987e-05, P150=1.3790322734959527e-05, P151=9.125482050483897e-06, P152=3.3670706537209717e-06, P153=3.5871609523740764e-06, P154=9.19714822062834e-06, P155=1.385992169737442e-05, P156=3.8557949611739195e-05, P157=8.477813974499851e-05, P158=0.00010953187086012318, P159=6.304258094841516e-05, P160=8.517828323261603e-05, P161=7.572238383957275e-05, P162=5.158708620703326e-05, P163=2.080929279541962e-05, P164=1.2363058016108324e-05, P165=8.540770511600291e-06, P166=1.2622284219028655e-05, P167=2.756524639061515e-05, P168=1.9848182633889888e-05, P169=1.0594906482431283e-05, P170=1.3627291625208814e-05, P171=7.173391498104004e-06, P172=6.948641991818958e-06, P173=1.4420185943769162e-05, P174=2.282382336870944e-05, P175=2.725655016021925e-05, P176=2.3770932257711774e-05, P177=2.368850757470672e-05, P178=1.1125079902858597e-05, P179=1.6349052893797922e-06),Row( P0=-640.1649994523534, P1=69.49198717170637, P2=-6.181559021136626, P3=2.545942471853354, P4=-9.989355614064255, P5=-6.541876543546155, P6=-2.2348384243571386, P7=2.6576348814191446, P8=1.0795853022118647, P9=-2.379902283149908, P10=-2.908927693089299, P11=-0.04885997665444499, P12=-0.9180702913904465, P13=-1.7218047570565713, P14=2.430822850648135, P15=-2.4028890188347694, P16=4.828606526698276, P17=4.7532731857844555, P18=0.9514403855286122, P19=5.494482633126295, P20=-1.3767271313730045, P21=3.0410019472208036, P22=0.5720667502057977, P23=0.08983883173706997, P24=-0.771226491353058, P25=0.06450325588685528, P26=3.31688659258404, P27=0.8506567554263246, P28=2.256107449125509, P29=2.0460450763047464, P30=-0.7223619591952544, P31=4.252138877890689, P32=1.3031839351163295, P33=3.4383737245057135, P34=2.6016072328204882, P35=2.4912530677862486, P36=-0.07485754634743422, P37=1.539637553971749, P38=-0.945912138924802, P39=1.0692687249021409, P40=0.7603491531479444, P41=0.700256224623975, P42=0.730439499368933, P43=0.7149241376996099, P44=0.6887527186428622, P45=0.6899277424352045, P46=0.6980944616705439, P47=0.6826281171409281, P48=0.718513001338198, P49=0.7309934295087526, P50=0.7564658174104213, P51=0.8185478460555226, P52=0.000698807689338482, P53=0.00034064064075939635, P54=0.0005551873496199176, P55=0.0002627705741250591, P56=0.00017977194939714896, P57=0.0010868514133527446, P58=0.0038632463421581684, P59=0.007710106711965379, P60=0.008162087709968508, P61=0.010038263459017, P62=0.008230236303082871, P63=0.0075140879770196735, P64=0.006660124168755938, P65=0.010836136742608232, P66=0.03021611011460005, P67=0.060540695845850645, P68=0.026196078233236664, P69=0.014969937737862857, P70=0.025305546724439677, P71=0.01321766997840102, P72=0.004157813700435043, P73=0.0051076753674282075, P74=0.009071989556572464, P75=0.01215061733331252, P76=0.008825042859260952, P77=0.019092314086345645, P78=0.03368940069430183, P79=0.0468841171733862, P80=0.0763023529794511, P81=0.06626462794820823, P82=0.04928706614462709, P83=0.03535476128749722, P84=0.025855703725478317, P85=0.014075626836873112, P86=0.010475263899213855, P87=0.015495121192844527, P88=0.025311789828742694, P89=0.029413690984774688, P90=0.036710070423176366, P91=0.026407860492046065, P92=0.025554693621956816, P93=0.011441597248856742, P94=0.017535898621167036, P95=0.01935296510794066, P96=0.018963426662210712, P97=0.06000044531991807, P98=0.04977882210645364, P99=0.029752700567010737, P100=0.013492866654442711, P101=0.006662243468710796, P102=0.004666886487231959, P103=0.006217770614373733, P104=0.0030773714626612775, P105=0.0028902062014446637, P106=0.0037777304816262354, P107=0.007042723906465864, P108=0.003759156051479555, P109=0.0033791203114954165, P110=0.007200755400990612, P111=0.007572895321912182, P112=0.003774855197644962, P113=0.0032106265614948353, P114=0.0012543448012028962, P115=0.0006896586104202818, P116=0.0007436456438845885, P117=0.0012712306021006158, P118=0.0007990152714473416, P119=0.0004104114602330126, P120=0.0005091480520624676, P121=0.0009963782783935264, P122=0.0007147880963909966, P123=0.0005718484615856419, P124=0.0005617224186034097, P125=0.0006413206108079298, P126=0.0004724173143745282, P127=0.000680952628728205, P128=0.0008235002401780743, P129=0.001219979744435202, P130=0.0006396657597627427, P131=0.00036900991789441776, P132=0.0003430328567904681, P133=0.0002672027614816981, P134=0.00013653063467959146, P135=0.00011656870039877942, P136=8.908053215738915e-05, P137=6.0051209418362376e-05, P138=8.082461356381102e-05, P139=5.7713560827915955e-05, P140=8.474188601706033e-05, P141=0.00010281055337470649, P142=0.00015338707970735305, P143=0.00011139150592326513, P144=0.00011303889273722628, P145=0.00016272510606226258, P146=0.0001139630372692253, P147=0.00010604961279833576, P148=5.853009732246016e-05, P149=9.195824377872684e-05, P150=0.00011562851476702238, P151=0.00011622492403678118, P152=4.338021476804896e-05, P153=1.435012461362835e-05, P154=1.3203592126923754e-05, P155=1.4375530407378113e-05, P156=1.8388695549064285e-05, P157=4.413859942850361e-05, P158=5.036019011745388e-05, P159=5.3960354116563646e-05, P160=7.901307937534875e-05, P161=8.333347886286301e-05, P162=2.6529729495700056e-05, P163=8.582337465881684e-06, P164=5.104946731158954e-06, P165=4.034018612524974e-06, P166=7.000885367857997e-06, P167=1.138341730801148e-05, P168=1.4411150549393891e-05, P169=9.612503512605498e-06, P170=5.507259663215759e-06, P171=5.15982056525684e-06, P172=3.171678357649082e-06, P173=4.766696289216563e-06, P174=6.509487855603225e-06, P175=6.832328751597375e-06, P176=6.728535684857833e-06, P177=8.159317680455778e-06, P178=4.2590945268319315e-06, P179=3.6194509593992046e-07),Row( P0=-755.3822322235065, P1=75.18824900440544, P2=0.08038943470256837, P3=9.145238259850196, P4=0.05210525432046274, P5=-6.3681604674662005, P6=-0.39433595054154014, P7=4.762759685442769, P8=3.5957475739428553, P9=-2.2486697544107086, P10=-6.794539586600845, P11=0.8250309181671182, P12=-0.01639806350479907, P13=-2.2306533857296085, P14=2.3975429438786255, P15=-1.8610003652632467, P16=0.0916631491984336, P17=2.783808328379931, P18=-2.772598649675866, P19=1.029192938539655, P20=-3.3307820348571333, P21=-2.128089847460403, P22=0.34566109386362537, P23=-2.377808834047615, P24=-2.168716625370747, P25=-1.008237988284035, P26=-2.468487437062176, P27=-0.19866738441928766, P28=-1.9373040389492908, P29=-0.5577752742871, P30=-0.7912057631649853, P31=0.08581588271349465, P32=0.7616959375448178, P33=0.5213189468741486, P34=0.7300448968824517, P35=1.6949710585972075, P36=0.9877216355638716, P37=3.2780494861418723, P38=5.000594390405767, P39=6.127155982515508, P40=0.7615615625606859, P41=0.715161740325698, P42=0.6944454068010061, P43=0.7447057820550742, P44=0.7534511785579053, P45=0.731230743347464, P46=0.7210357897880312, P47=0.7202702383735134, P48=0.7685632263068132, P49=0.7758535366340533, P50=0.7424041346871785, P51=0.7490106974157341, P52=1.555736375590155e-05, P53=7.88913834113119e-06, P54=5.657199985764291e-06, P55=1.6782323238279353e-05, P56=0.00028412244385321036, P57=0.0040394735939433485, P58=0.005850656348738941, P59=0.0034122568922841327, P60=0.0016364211965426813, P61=0.00040527690219112084, P62=0.00039877778415668355, P63=0.0018283229098741338, P64=0.004791179300302108, P65=0.00608267411047527, P66=0.004003328984582049, P67=0.0021953655675262473, P68=0.002107732740870539, P69=0.0018261759672861212, P70=0.002548166276855359, P71=0.0025747314605667417, P72=0.0016409932721415544, P73=0.002099036403862133, P74=0.001488803017512882, P75=0.0009510242826685622, P76=0.0024031392653862506, P77=0.0068426686134639555, P78=0.004479388900597111, P79=0.001980185698264827, P80=0.0013789379067696787, P81=0.0011870604466658885, P82=0.001758097911698224, P83=0.004512993431359944, P84=0.007603205163879639, P85=0.00675584892321462, P86=0.0026992954081368104, P87=0.0024158872095370186, P88=0.006143157111174611, P89=0.004648242222031443, P90=0.004892327218978437, P91=0.0034093131287045797, P92=0.0069336662724327185, P93=0.005923361943078187, P94=0.0027382566283362074, P95=0.0012136125972264616, P96=0.002014647503824674, P97=0.006545661992889885, P98=0.002626024296309885, P99=0.0002063019423213681, P100=0.0002574674566027829, P101=0.00018717748798784906, P102=5.4650649739367256e-05, P103=5.286123390303208e-05, P104=7.53665718289287e-05, P105=0.00012881448221294407, P106=7.630778749851991e-05, P107=0.00019904433346830074, P108=0.00017156918818903185, P109=9.028665799828476e-05, P110=0.0011955645564033592, P111=0.0017113714901451347, P112=0.0002151358334728323, P113=0.0001655187181998646, P114=0.00028048136984423836, P115=0.0001607636921452246, P116=0.00027504506001462084, P117=0.0003712360552544613, P118=0.00022303654006416274, P119=0.0001330410881721919, P120=6.793112392952036e-05, P121=8.867812607106517e-05, P122=8.510114054658434e-05, P123=0.0001891731471210073, P124=0.00021483889442006093, P125=5.818393709677408e-05, P126=7.27453740341023e-05, P127=4.645153250408342e-05, P128=2.5100233454305218e-05, P129=2.98417829218534e-05, P130=3.77564942090187e-05, P131=2.751956938655747e-05, P132=6.006473663977871e-05, P133=4.683459352476579e-05, P134=3.697893027180579e-05, P135=1.9400722264306093e-05, P136=2.1652897804788035e-05, P137=1.0135751856382495e-05, P138=1.283371007607138e-05, P139=1.4260402837810333e-05, P140=1.7520390832445456e-05, P141=1.73677937056368e-05, P142=1.9042344710197e-05, P143=2.8936720786479265e-05, P144=4.4701252888843375e-05, P145=3.90186196768578e-05, P146=1.1410985295602496e-05, P147=7.037547484016998e-06, P148=3.4988496972644865e-06, P149=3.7406785181379243e-06, P150=2.8560192864572073e-06, P151=1.36250935735308e-06, P152=1.2139641322653789e-06, P153=2.106287829036001e-06, P154=2.591795669322092e-06, P155=5.010219628076086e-06, P156=5.279846215541658e-06, P157=3.886608215128728e-06, P158=4.60328462628861e-06, P159=5.346847597783849e-06, P160=3.093253609972338e-06, P161=1.3541880180135153e-06, P162=1.3931152135835973e-06, P163=1.3092805902553402e-06, P164=1.1651256177715608e-06, P165=1.1115723901588166e-06, P166=9.417158058239768e-07, P167=6.957019391011209e-07, P168=8.104881883105368e-07, P169=6.785432579408534e-07, P170=4.531444998824019e-07, P171=5.910499259459312e-07, P172=3.369256874227712e-07, P173=2.4045020200448054e-07, P174=4.4552868932894885e-07, P175=7.753797888860734e-07, P176=9.201271852676953e-07, P177=1.435755931111159e-06, P178=7.030362019475751e-07, P179=6.361402859068498e-08),Row( P0=-738.1453323188568, P1=86.79919208173405, P2=-0.016815576532265723, P3=11.197430133565305, P4=-2.307954060268785, P5=-6.854439272613982, P6=-0.6781602908513348, P7=5.1474947429261615, P8=4.8334035458457, P9=0.3649394763515152, P10=-2.8419993981594085, P11=0.9457963889590008, P12=-1.6463011496164652, P13=-0.3598396733448154, P14=5.89283400076515, P15=-3.5355380137380874, P16=1.961238914566705, P17=3.1937418653192386, P18=-1.5456311252474069, P19=4.508850418603714, P20=-2.2027059819210284, P21=-1.412304778540454, P22=0.3381561129772064, P23=-1.9875198925086042, P24=-0.9901929458862868, P25=-0.825823349320211, P26=-0.8936304287569155, P27=1.623964978403744, P28=0.9503495125257763, P29=2.2082518914128015, P30=0.7624073682459594, P31=2.3496273728766357, P32=1.3875382265237528, P33=0.9072397664133277, P34=0.5223111944179453, P35=0.5796557479765567, P36=1.1188849092237008, P37=3.5969559769479136, P38=4.748253761269627, P39=3.908626174468191, P40=0.7900321125954207, P41=0.7437952903005407, P42=0.7501302977652141, P43=0.7697532967984397, P44=0.7420030676225268, P45=0.6902173231175532, P46=0.6766384962590442, P47=0.6752872502579099, P48=0.7588640293365078, P49=0.7526227006337816, P50=0.7438680963606247, P51=0.7812782164419932, P52=3.539633291088363e-05, P53=1.8950989100277983e-05, P54=4.5788212907385545e-05, P55=0.00017348723487657238, P56=0.0002836762541131556, P57=0.0017925540264139923, P58=0.003164760404095182, P59=0.0017311343159887584, P60=0.0012204239964323407, P61=0.0017805297288522788, P62=0.002291508904230681, P63=0.001464168427793434, P64=0.002512638884097075, P65=0.004303744623928556, P66=0.0019728232003404654, P67=0.001353275027654278, P68=0.001309177370520027, P69=0.0018336221194402657, P70=0.002768480008820792, P71=0.0029849668146781317, P72=0.0030791739308563535, P73=0.002462993878860218, P74=0.0009051631391695749, P75=0.0008139439483156647, P76=0.001423653727379361, P77=0.006456451695472963, P78=0.010235279494717182, P79=0.007165795149529716, P80=0.0030639200583479466, P81=0.0014546978622452303, P82=0.002354065520691371, P83=0.0073402235948000485, P84=0.014411424876142416, P85=0.016120652636955682, P86=0.003918459217698502, P87=0.001228208632133576, P88=0.001476034770752077, P89=0.0027200442912438632, P90=0.008606932120049605, P91=0.005703880041157851, P92=0.002722518999422622, P93=0.0012340875795850352, P94=0.0013115903500314557, P95=0.0025087353618009826, P96=0.0017285407791589752, P97=0.0005208471238875241, P98=0.0006555264933833166, P99=0.0006337503400383364, P100=0.00023992854122416088, P101=8.829490387143269e-05, P102=9.083235818561318e-05, P103=0.00012452023035978304, P104=0.00013078220255188268, P105=0.0006712053852707053, P106=0.0007861033077538914, P107=0.00026526639725387775, P108=0.0002432763569208294, P109=0.00021044912590363876, P110=0.00047994431893612284, P111=0.00038523405226276565, P112=0.00037453821884107205, P113=0.00022182330943000306, P114=0.00013273856459155973, P115=0.00013242786592560394, P116=0.00017167096159237608, P117=0.00026440435626928625, P118=0.00018116296094744026, P119=0.00011492992912421154, P120=0.0001337292892147987, P121=7.37995700811276e-05, P122=6.932363226870964e-05, P123=0.0001653063313291213, P124=6.978030376056618e-05, P125=8.035839372480888e-05, P126=0.00020107787858127146, P127=6.0330851447979e-05, P128=5.941940965088523e-05, P129=5.0642813438365454e-05, P130=5.3904374353689524e-05, P131=3.4820593378060116e-05, P132=2.0035863189267155e-05, P133=2.152031775760068e-05, P134=1.6993404628636676e-05, P135=1.1304885510976694e-05, P136=8.526409627879647e-06, P137=4.402012678317394e-06, P138=3.7031812602105065e-06, P139=4.3775604789222345e-06, P140=4.469451098314891e-06, P141=6.871645369661181e-06, P142=6.242194552327545e-06, P143=1.0481212318937038e-05, P144=1.8663327782598773e-05, P145=3.159917288248699e-05, P146=2.453967776324486e-05, P147=5.689461140227898e-06, P148=2.6020520475360324e-06, P149=3.0905109886065852e-06, P150=2.085907118697264e-06, P151=1.4181333488860931e-06, P152=9.128019507533382e-07, P153=1.0712537355384256e-06, P154=1.8889207668252002e-06, P155=1.220981124948893e-06, P156=1.2015168967625816e-06, P157=1.9203999736167044e-06, P158=1.4748749251665579e-06, P159=2.0156091870005986e-06, P160=1.3540224819823086e-06, P161=9.846722091250015e-07, P162=6.792647259533667e-07, P163=2.9835143961385177e-07, P164=2.989174083277211e-07, P165=5.49474265841794e-07, P166=6.407420363954865e-07, P167=6.740752995327777e-07, P168=3.3793802878319346e-07, P169=1.728062609022329e-07, P170=1.3979154866235776e-07, P171=1.327233675175934e-07, P172=1.4338192015591987e-07, P173=2.273787793816105e-07, P174=2.2552549464430746e-07, P175=1.2904491539548938e-07, P176=3.49958254960392e-07, P177=4.7445167657040274e-07, P178=3.116326450626003e-07, P179=2.2331564570996058e-08),Row( P0=-736.5192646602449, P1=64.70134836713035, P2=-8.215652050528359, P3=6.516494163577579, P4=-4.22569503231851, P5=-6.794196820470988, P6=-0.5994612861622874, P7=1.4321359927709474, P8=5.853627030942522, P9=0.5149662179646497, P10=-6.118000605214502, P11=0.9661058611583958, P12=-1.262043665164611, P13=-1.107093122925132, P14=1.7349404200370042, P15=-3.4045602835982156, P16=0.9269410629134099, P17=1.4148412368214436, P18=-2.339088577912681, P19=2.9811399651984036, P20=-3.7492708644588646, P21=-2.1879197073184873, P22=-0.03718793851241449, P23=-2.3812939637911796, P24=-2.7510180479071558, P25=-0.74092266613226, P26=-2.3049521214137756, P27=-0.12247660591863789, P28=0.21006866667493296, P29=0.4483613405010309, P30=1.308720130322765, P31=3.187807194968303, P32=2.5502330605122245, P33=1.6631831022102916, P34=1.7237410356116198, P35=2.466396650000991, P36=1.7329421578847437, P37=4.448691022072166, P38=5.0623255560601645, P39=4.436501364851516, P40=0.7396798823267137, P41=0.7366895871105976, P42=0.7033603817262521, P43=0.7035869445461106, P44=0.7405490265342789, P45=0.719678098393055, P46=0.7043941607461989, P47=0.6830937664510613, P48=0.7122843554138578, P49=0.7677407409361179, P50=0.7351960862594852, P51=0.7226957860248819, P52=1.0440061443369672e-05, P53=4.205969680314863e-06, P54=1.657812654982977e-06, P55=4.758635833426518e-06, P56=0.00010311799726428856, P57=0.0014216887371483536, P58=0.002978084038372442, P59=0.003550151493051672, P60=0.003044315582977498, P61=0.0007720212242960751, P62=0.0003237245426707523, P63=0.0006704029590932872, P64=0.0026078416111209412, P65=0.006248923034999567, P66=0.011462142921571715, P67=0.011587300074801736, P68=0.005798499999016846, P69=0.004172400483540646, P70=0.0020062655047321165, P71=0.0017734514122531358, P72=0.002210496610464905, P73=0.0015635909381688671, P74=0.0023817163773226777, P75=0.0028164266183204204, P76=0.002191379029978178, P77=0.005201490679099968, P78=0.009463396361829109, P79=0.011271513031187277, P80=0.005479023836153088, P81=0.0026184412939907783, P82=0.0023848566716033032, P83=0.0028125816113861394, P84=0.005217987029177222, P85=0.016012453608628904, P86=0.01914845679714245, P87=0.005939037733775999, P88=0.0022353471928175635, P89=0.004280141822549688, P90=0.0075539288811036645, P91=0.010863111540199932, P92=0.017049189194649678, P93=0.027999911329282753, P94=0.008862013061887365, P95=0.001343239260443377, P96=0.0018818271628328067, P97=0.0004638445454390302, P98=0.002407195445847451, P99=0.003843530171457411, P100=0.0005789614139679452, P101=0.00012491878225075483, P102=4.3647141519219457e-05, P103=0.0004187182889601662, P104=0.00044200482307254184, P105=6.449960279628686e-05, P106=0.00027566158174332947, P107=0.0022188561254004565, P108=0.002003676961743298, P109=0.00038604491815070563, P110=0.0014611085362300998, P111=0.0033247985697502517, P112=0.0032861780271858914, P113=0.0004974055265277229, P114=0.0003322220508497716, P115=0.0005532379639751985, P116=0.00034458899320991834, P117=0.0009236212535877618, P118=0.0003737315228378474, P119=0.0001432000927533522, P120=0.0001415918896961769, P121=0.00016135242073455854, P122=0.00020990634406265645, P123=9.181776231071608e-05, P124=4.793008877774163e-05, P125=5.8189850088580564e-05, P126=5.13477846297819e-05, P127=8.44144022036488e-05, P128=4.4920953618645526e-05, P129=4.6363570654565814e-05, P130=3.64825675580143e-05, P131=3.1050266581581e-05, P132=6.230926625332394e-05, P133=8.610575976029003e-05, P134=8.312432289975622e-05, P135=2.2602555065712646e-05, P136=1.5581746266517276e-05, P137=4.489491119024329e-06, P138=6.405890792692056e-06, P139=6.448048989625493e-06, P140=8.502138591144679e-06, P141=1.1572440844959169e-05, P142=1.764592580173798e-05, P143=3.7799427915206594e-05, P144=7.057922522848715e-05, P145=6.388219890891815e-05, P146=5.606221808069287e-05, P147=2.510584616184767e-05, P148=8.049320690337092e-06, P149=5.104418650588867e-06, P150=4.443609140821085e-06, P151=6.755459527041187e-06, P152=1.6060210637072954e-05, P153=1.2027092525307972e-05, P154=7.112835724048418e-06, P155=7.961841370778328e-06, P156=9.146540707337325e-06, P157=6.443442114283386e-06, P158=6.3202151107234085e-06, P159=4.6914744942465416e-06, P160=5.449524636441976e-06, P161=2.8749284869379374e-06, P162=6.648547733656077e-06, P163=3.7533559183151554e-06, P164=3.0186589792721775e-06, P165=3.9326659239897315e-06, P166=3.968164851608331e-06, P167=3.1225572607054606e-06, P168=5.540497174974205e-06, P169=5.694916887057709e-06, P170=3.779742016026203e-06, P171=3.130410715302266e-06, P172=1.9507254964299408e-06, P173=1.5188424736283777e-06, P174=3.0512049036094366e-06, P175=3.897522951100636e-06, P176=4.464442414964408e-06, P177=2.6358818903500895e-06, P178=1.0699002838058335e-06, P179=1.2672752029293216e-07),Row( P0=-655.7713317855498, P1=52.911011732985344, P2=-1.505691635807797, P3=3.7975488560121735, P4=-4.520899604380565, P5=-6.80434634769283, P6=-2.843788417371912, P7=1.8092610122046389, P8=-2.756034043942677, P9=-0.3347866526113239, P10=-5.467212098168494, P11=-0.7071974504363244, P12=-0.42794355420682817, P13=-2.7301628719055278, P14=2.848640316824336, P15=-4.2432056179498145, P16=-0.6754825466131879, P17=0.8901237258808078, P18=-1.8615622663067388, P19=1.203732184943, P20=-4.64572382198533, P21=-0.8509053933508891, P22=1.418397461980805, P23=0.4342269711104971, P24=0.9431449733088398, P25=3.680488142137852, P26=1.86251104162127, P27=4.286378528789244, P28=2.945735196261064, P29=4.661510658268771, P30=2.530321055154848, P31=5.014724423140045, P32=2.879751243301262, P33=2.995666782079776, P34=1.4661402754615207, P35=1.8030789556288789, P36=-0.24352403725886235, P37=1.6675625758941626, P38=2.1471897275313947, P39=1.0412427266877402, P40=0.7662439836744296, P41=0.7235364626545538, P42=0.7541055314709659, P43=0.7865075857718433, P44=0.7735715733689214, P45=0.7480683152622907, P46=0.6668996002550178, P47=0.6719771775879817, P48=0.6703099775223044, P49=0.662026655431417, P50=0.6886559355798167, P51=0.7554047605979765, P52=0.00013876243250484776, P53=4.5132870356830024e-05, P54=3.085431046422657e-05, P55=4.463121431689531e-05, P56=0.000138191148781482, P57=0.0007866769194085571, P58=0.0021344535138538407, P59=0.010811831007785919, P60=0.022315825633787596, P61=0.03350465066568727, P62=0.03858876776123312, P63=0.011092160551365898, P64=0.0010147432418629435, P65=0.0012741664355049166, P66=0.004732079170836539, P67=0.010259723797429298, P68=0.0148017432713035, P69=0.020223695939757435, P70=0.028681186852640483, P71=0.026904249351136326, P72=0.03513798333361552, P73=0.07207736984739743, P74=0.03996170627306035, P75=0.00441290506371022, P76=0.00470125975304336, P77=0.010352000306931538, P78=0.015160734637539034, P79=0.013941849455300293, P80=0.010226769775551746, P81=0.015857104979638825, P82=0.029987026208900733, P83=0.07345990423430378, P84=0.09443342238729167, P85=0.0353234478500518, P86=0.008955333238059667, P87=0.01273615595515379, P88=0.019204715531176895, P89=0.03947732823466803, P90=0.04850968337979036, P91=0.01574651795074064, P92=0.017602695706304558, P93=0.01894506349026099, P94=0.01441564890072064, P95=0.00769024821861647, P96=0.06180643167148951, P97=0.07111165249168978, P98=0.007827004635941949, P99=0.0038310869724824557, P100=0.0015650870794178952, P101=0.0011204845829288054, P102=0.0028074007322330756, P103=0.0011782382957374212, P104=0.0006941765043939044, P105=0.0015434043630584186, P106=0.004236403937338696, P107=0.006359499015216557, P108=0.0010516615532351838, P109=0.0007911744805488443, P110=0.004394095924146184, P111=0.010143551815066206, P112=0.0031722329788957475, P113=0.0004816883541952153, P114=0.0005029365909523815, P115=0.0004407556743597612, P116=0.0005050288805351094, P117=0.0003172538712960808, P118=0.0004922683243648048, P119=0.0003345319599595999, P120=0.00029340863268646436, P121=0.0005377920361959347, P122=0.0004076712486856957, P123=0.0008495546057943269, P124=0.0006778656028575369, P125=0.0004929109485311933, P126=0.0009374902653846474, P127=0.00043722253811458395, P128=0.0003497151948735618, P129=0.00030336582185642714, P130=0.00034130267102678495, P131=0.00024481551685486577, P132=0.00034109828998016463, P133=0.0003304328335289109, P134=0.0006340370512342493, P135=0.00024262393797497324, P136=0.0002772610191646068, P137=6.922825325838737e-05, P138=6.097466386464939e-05, P139=8.743240169826466e-05, P140=8.586751589676035e-05, P141=8.348309678904836e-05, P142=0.00010329427747771916, P143=0.0002064170832939888, P144=0.0003307486375434498, P145=0.0002686575522225364, P146=0.000140243466281909, P147=0.0001334835089014476, P148=6.733484331534766e-05, P149=3.753865088550035e-05, P150=3.6587721074429035e-05, P151=3.043716762420286e-05, P152=6.728214030692408e-05, P153=5.463114868163698e-05, P154=6.18809580018127e-05, P155=9.907502302144942e-05, P156=0.00013129168570354, P157=0.00014062430163016263, P158=0.00019542321728887005, P159=0.00013362281326188862, P160=0.0001527036871891849, P161=9.652041036940857e-05, P162=6.891796034235732e-05, P163=4.24194564885039e-05, P164=6.142789586461343e-05, P165=0.00010002958301839715, P166=0.0001042032448420755, P167=0.00011514343335604624, P168=9.626320064924743e-05, P169=5.002500792249983e-05, P170=3.636029743355337e-05, P171=2.1846853094236458e-05, P172=3.100335027153548e-05, P173=9.155817102301885e-05, P174=0.00012158479736622778, P175=0.0001379585123927151, P176=8.042719345158248e-05, P177=8.156561315088542e-05, P178=3.8897260486943346e-05, P179=4.754975700113184e-06),Row( P0=-647.3572419577704, P1=59.17740477530079, P2=-2.586520069604992, P3=4.60621078035869, P4=-4.008938147329341, P5=-3.1469072647596037, P6=0.013178046672136443, P7=6.2825495336837704, P8=0.3089895176535411, P9=1.241401793466021, P10=-6.110152057869897, P11=1.2794702058716247, P12=0.8452421421519325, P13=-1.4391435275506836, P14=3.837498171059909, P15=-1.9542329665579092, P16=1.4796979914593957, P17=4.149035680608784, P18=0.06216768935263356, P19=4.445739883781841, P20=-1.8625831949632798, P21=3.1664072170849056, P22=5.0567371910629815, P23=2.9547232457098405, P24=3.422067450540767, P25=5.868044250319037, P26=2.8268879832372167, P27=5.179246169237042, P28=3.297186707590357, P29=4.903070101725997, P30=1.4679985411267031, P31=4.760236386949412, P32=2.6436421099236567, P33=3.4221935189061434, P34=0.6718228416896416, P35=0.7032004405303978, P36=-1.4315936239983247, P37=1.122453987392483, P38=0.9789516299447371, P39=1.0131042549955689, P40=0.7280211190336865, P41=0.7298769981181492, P42=0.7128370086928434, P43=0.7368520569349715, P44=0.7849423337082856, P45=0.7907601888972676, P46=0.7820994177143987, P47=0.7170083898893117, P48=0.6765944223840883, P49=0.6651993314398363, P50=0.657902543404485, P51=0.6955974608763634, P52=0.00029626392887352926, P53=0.00039152401554509345, P54=0.00013083143145739014, P55=4.2791526971252385e-05, P56=0.00011297931142009409, P57=0.0005420748890002536, P58=0.0013422093552461275, P59=0.0062313274560345105, P60=0.012109013823182174, P61=0.027107251425968253, P62=0.03685941044128135, P63=0.022344719096786047, P64=0.009081470758267011, P65=0.0031214284283556633, P66=0.0032871404967765314, P67=0.008378564425805738, P68=0.008651961222163584, P69=0.008782089132150065, P70=0.014973091157395447, P71=0.017984306658596128, P72=0.02543186876793429, P73=0.030871537456992375, P74=0.022439413274235324, P75=0.013229348592864185, P76=0.02117993162138248, P77=0.050708740694983094, P78=0.04541300971746737, P79=0.021473969011013266, P80=0.00795780523528288, P81=0.022452359706102102, P82=0.08792913345065119, P83=0.07211379620719577, P84=0.05248922766929292, P85=0.08079497732444106, P86=0.02653651283016284, P87=0.009568195821733978, P88=0.021392289648734793, P89=0.027938909794909997, P90=0.028097580384338565, P91=0.028135905022931253, P92=0.038932441735082327, P93=0.03243933709194543, P94=0.0474055491897923, P95=0.029300893019890653, P96=0.010419843589909236, P97=0.015174242820882292, P98=0.021915611328324397, P99=0.01779337627903175, P100=0.005841057609661363, P101=0.003062715175014438, P102=0.0014121554489203354, P103=0.0007551263337848945, P104=0.001090942520190489, P105=0.002268085231663814, P106=0.0031084706440177826, P107=0.006287288987594674, P108=0.0022206054103629377, P109=0.0006885446618987841, P110=0.003568849145316607, P111=0.006609349730547479, P112=0.005706359956945936, P113=0.001918668304038057, P114=0.0027656085753945485, P115=0.0010336756388415445, P116=0.0017579915948953849, P117=0.002969735089275617, P118=0.0011406138918173027, P119=0.0007171548981837451, P120=0.001038788745066365, P121=0.0009080882522329805, P122=0.000847381483124913, P123=0.002222512577059604, P124=0.000838878794025218, P125=0.0004982328844581769, P126=0.0010151574961397508, P127=0.0012090693137309823, P128=0.0010261713627801269, P129=0.0006649717430758071, P130=0.0005428003452134479, P131=0.0005427137652490306, P132=0.00036564727765553884, P133=0.0005252867291404414, P134=0.0004959758594441952, P135=0.0003080495674158845, P136=0.0004393645757510673, P137=0.000527897962584255, P138=0.00012480232853130383, P139=0.00014710899356057205, P140=0.00012278242614551535, P141=9.412960058149946e-05, P142=0.00013846785737990915, P143=0.0002882550822322548, P144=0.0004508025241119013, P145=0.00026100419842216483, P146=0.00021280683785915102, P147=9.350370470240986e-05, P148=7.564220820324557e-05, P149=2.6231784883501884e-05, P150=1.2761885317249102e-05, P151=7.099134108668779e-06, P152=1.2351430984799001e-05, P153=2.238784582866019e-05, P154=3.858376508819831e-05, P155=0.00010812383786101526, P156=0.00010031629466988348, P157=0.00017516163498384723, P158=0.00026052733013364837, P159=0.0001314056834561522, P160=0.00013044776024348628, P161=9.657994806243667e-05, P162=4.943748809630641e-05, P163=4.327579911281692e-05, P164=4.5367104853790047e-05, P165=6.660809467977847e-05, P166=0.00013381081134572476, P167=0.00011197114074728114, P168=9.530686166152017e-05, P169=9.6812009332604e-05, P170=7.243968478426567e-05, P171=4.596654118880538e-05, P172=4.128838384873115e-05, P173=9.541869347966929e-05, P174=0.00012973540238634976, P175=0.00014706296190798208, P176=9.790951361870943e-05, P177=9.47588247494135e-05, P178=3.9170387932334055e-05, P179=5.716959520237134e-06),Row( P0=-684.9016121154997, P1=71.05869428134557, P2=-5.995670214018625, P3=1.7874348437956864, P4=-6.268607332510548, P5=-4.043021038868667, P6=-1.289925192102634, P7=3.143777106185621, P8=1.4226841968719757, P9=0.09451909029821948, P10=-3.995200319624557, P11=-1.8343778951501386, P12=0.43445063641122705, P13=-3.655230291470076, P14=2.2177895576585214, P15=-2.885914870848786, P16=1.0471056054781256, P17=1.5563968747904136, P18=-0.5490692979591463, P19=1.6027139269467146, P20=-4.002605230926987, P21=-1.8662512354699807, P22=-0.5175684915776126, P23=-0.7864778487899616, P24=-1.8021305036557915, P25=1.013744669336383, P26=-0.2796651448115889, P27=2.2105056131099583, P28=1.201183034227627, P29=1.4655732366103076, P30=0.5286867546338114, P31=1.7799152835181475, P32=2.2138671221607544, P33=3.745584320359819, P34=3.850320974665796, P35=3.9496230784674538, P36=3.296898038255925, P37=2.6342095542696313, P38=1.0024131258198843, P39=2.21817200464913, P40=0.7605774705568281, P41=0.7228578295220784, P42=0.7363156979923493, P43=0.752416141598501, P44=0.7483375033434609, P45=0.7329084452852326, P46=0.7334333031514525, P47=0.73047894098258, P48=0.7096380580167908, P49=0.7006251254951362, P50=0.7663930977605469, P51=0.7918713643883757, P52=6.631993055196628e-05, P53=2.5587398374330395e-05, P54=2.136767107954671e-05, P55=5.602758632945768e-05, P56=0.0002672533803335186, P57=0.002045996028518644, P58=0.004570906840457076, P59=0.006928587593630995, P60=0.007661746015599508, P61=0.0065453127795110045, P62=0.002631378521775632, P63=0.0013091275474498307, P64=0.0016660094105918506, P65=0.006511308570807276, P66=0.008914021478507823, P67=0.007912452043293954, P68=0.008528471333045356, P69=0.012842212433137995, P70=0.011225723124940532, P71=0.00565642616932377, P72=0.0031981729780084438, P73=0.004108436893788725, P74=0.003558909752980258, P75=0.0033078627758708276, P76=0.0036149541192235474, P77=0.0031231362598778427, P78=0.005199028460049911, P79=0.012174750344390332, P80=0.01825510438973303, P81=0.015474414669296535, P82=0.01293625420388278, P83=0.009253762967810242, P84=0.009826706930145826, P85=0.006056355475546044, P86=0.005406201047173992, P87=0.0070248329909728055, P88=0.013818749465210644, P89=0.019298250283685727, P90=0.03167005436661671, P91=0.03276424307990027, P92=0.011119290021582781, P93=0.008391862535292397, P94=0.013127958240598814, P95=0.023204573187863568, P96=0.03048003483289581, P97=0.04260486610200923, P98=0.02946102880679405, P99=0.0033698846495762366, P100=0.00202820010234321, P101=0.0018134202707747829, P102=0.0014117426469283832, P103=0.004103525884492282, P104=0.001674404052252045, P105=0.0029416631159427647, P106=0.003811047962673889, P107=0.001909934621358283, P108=0.0007967905771892917, P109=0.0017517101455404803, P110=0.0013628216337421542, P111=0.0014639577271150681, P112=0.0009111831763466535, P113=0.0004002902950203022, P114=0.0005936314114675723, P115=0.0009231322441348338, P116=0.0008715968920786736, P117=0.00035059637234222265, P118=0.0007626548758480274, P119=0.0002976937123065332, P120=0.0007511210512493442, P121=0.0006872398831945315, P122=0.0005186087690445293, P123=0.0008593321505299471, P124=0.00019133479125619715, P125=0.00028690659983061836, P126=0.00013441371542726124, P127=0.00015170573696524634, P128=0.00013591837732296528, P129=0.00020805083768283127, P130=0.00011743054749049617, P131=8.207478794091264e-05, P132=8.963053568417686e-05, P133=0.00012660832382561756, P134=9.465992231220345e-05, P135=0.00019581381365213002, P136=7.84746368679935e-05, P137=3.040772327612686e-05, P138=3.257121215722826e-05, P139=1.7223179754843352e-05, P140=3.094793582024942e-05, P141=4.1329867070913715e-05, P142=5.5878243745531824e-05, P143=7.57701374161355e-05, P144=7.772126803260606e-05, P145=5.0952775565093866e-05, P146=3.0410558434253174e-05, P147=1.5324039134739067e-05, P148=1.0677299659708288e-05, P149=9.690630544443038e-06, P150=5.761376547849367e-06, P151=6.240977836590996e-06, P152=6.210231573803649e-06, P153=1.57873089266053e-05, P154=2.0076091378713246e-05, P155=3.4001950411837584e-05, P156=6.83367932188254e-05, P157=0.00010159761148868217, P158=8.255213736522296e-05, P159=6.44589895046878e-05, P160=5.362262588149361e-05, P161=5.507861071342594e-05, P162=5.801844191951725e-05, P163=4.67748591403943e-05, P164=2.3343065667387963e-05, P165=2.6758394347611173e-05, P166=2.951930390017134e-05, P167=1.7672365177199958e-05, P168=1.3288418891615063e-05, P169=1.272054212632214e-05, P170=1.7077182163776928e-05, P171=8.234716653221553e-06, P172=7.490688291784175e-06, P173=6.5790370714919656e-06, P174=1.1080772033514098e-05, P175=9.960785307442638e-06, P176=1.3179897248533838e-05, P177=1.4565263082822263e-05, P178=9.188699827279444e-06, P179=9.93559204099202e-07),Row( P0=-725.4682586908856, P1=66.35484730637155, P2=-1.0784321360832776, P3=1.8283687498190704, P4=-5.878523796662567, P5=-3.6663845401621593, P6=-0.7406347212732453, P7=4.056220020302466, P8=-0.7763401995809809, P9=-1.4020071576072073, P10=-3.376662792846643, P11=-1.9015610668624918, P12=-0.5237052432430421, P13=-2.720803406126129, P14=0.8991246055624911, P15=-3.1280119347892437, P16=1.147621781895915, P17=0.7949458712077404, P18=-1.5861765684999871, P19=1.446615998826945, P20=-5.2164704089497915, P21=-0.2254347816206737, P22=0.5967153352149375, P23=-0.6794084519161666, P24=-1.1070881539505673, P25=1.967709662456386, P26=1.1614347736050823, P27=2.6420186152162777, P28=1.4210420495548046, P29=1.3192848381991509, P30=-0.13049861647797004, P31=3.3545698598633074, P32=1.6606064271400198, P33=2.519670611143944, P34=2.2882323119495473, P35=3.2208724663227426, P36=2.4482498018406247, P37=3.439518725633979, P38=2.819579576759301, P39=2.6382074183540056, P40=0.7136029312826908, P41=0.7555521655087196, P42=0.7764112400113311, P43=0.7678906386096007, P44=0.7921767044070087, P45=0.7529175177590461, P46=0.713102284954311, P47=0.7216759037598117, P48=0.7324086025650585, P49=0.755150248104044, P50=0.7747473850188445, P51=0.7702113709211852, P52=2.0406735423649683e-05, P53=8.315934020701879e-06, P54=1.0212976584676067e-05, P55=3.328546796620179e-05, P56=0.00026211265886042444, P57=0.002652411103687081, P58=0.00472269918529172, P59=0.0042652588293793075, P60=0.0044246260891819275, P61=0.005968027510616218, P62=0.005012602578074039, P63=0.001575150462443732, P64=0.0014586305953833438, P65=0.005387509327487314, P66=0.006951967982697388, P67=0.005132555335074391, P68=0.0033943752158026048, P69=0.004552237397563964, P70=0.007122568957724938, P71=0.004267864710429876, P72=0.004611680546853764, P73=0.004780839713007693, P74=0.0029194610600457097, P75=0.002119867442663056, P76=0.0023534075628645137, P77=0.006656567896654948, P78=0.008264057741521356, P79=0.013950424402243136, P80=0.014092222880362623, P81=0.009629873213998226, P82=0.012309642510522049, P83=0.010983823006813017, P84=0.0068105471414614555, P85=0.005284113450800923, P86=0.007579348639176918, P87=0.006844010183612368, P88=0.0037664081285051967, P89=0.002091157641481792, P90=0.004030079744462429, P91=0.007426407383782256, P92=0.01332809582460193, P93=0.004978739374332179, P94=0.0019423680578201101, P95=0.0015406069603140747, P96=0.002506548070221453, P97=0.005741867743743609, P98=0.0038428389239603773, P99=0.0019620607473665222, P100=0.0007099580312572445, P101=0.0007595940123346998, P102=0.00048055495524136757, P103=0.00026798914703724013, P104=0.00030751776543640603, P105=0.00031715524894255836, P106=0.00041753550523242525, P107=0.000724625907763945, P108=0.0004633175691319478, P109=0.0006883493778509253, P110=0.000995005342312576, P111=0.000989820471138975, P112=0.0003384982015364239, P113=0.00028219743036462104, P114=0.000104692528689108, P115=7.278836946203222e-05, P116=0.0002036293242346073, P117=0.0001228507942959901, P118=0.0002314705921092195, P119=6.744771570982605e-05, P120=0.00021066923590577823, P121=0.00014827553901387866, P122=0.0003232268780427988, P123=0.00017806595444283622, P124=0.0006132545680147536, P125=0.0002892072153503944, P126=0.00017972591061716508, P127=9.924079784047265e-05, P128=8.6105941539163e-05, P129=7.377989030911334e-05, P130=8.105493939538665e-05, P131=5.121190514155805e-05, P132=0.00010683310639698001, P133=0.00010911528736454849, P134=0.00011194008589952059, P135=9.860003799681996e-05, P136=0.00012572704132759037, P137=1.8356629594707672e-05, P138=1.4558444066676578e-05, P139=1.3223066883521948e-05, P140=2.010456029027207e-05, P141=2.6116999113535088e-05, P142=2.906117407200576e-05, P143=3.0439849972878093e-05, P144=5.3782828814925736e-05, P145=7.588243358668971e-05, P146=5.1169429742319195e-05, P147=3.9542998148143554e-05, P148=2.052749136795554e-05, P149=1.5242352444961397e-05, P150=1.6234318948504002e-05, P151=1.947297622304274e-05, P152=9.138608725964378e-06, P153=2.4327277862723044e-05, P154=4.555795585104895e-05, P155=8.718091945109373e-05, P156=8.502157358242733e-05, P157=0.00012025235882379117, P158=0.00015606872203220725, P159=0.00014729742817177875, P160=0.00017117023373699238, P161=0.00015284167279541231, P162=0.0001066845719679475, P163=5.9112554814148236e-05, P164=3.492420385982178e-05, P165=3.691304898518113e-05, P166=3.567448773276048e-05, P167=5.185911326372334e-05, P168=4.268253401937224e-05, P169=2.4554630879071754e-05, P170=2.5360057087301657e-05, P171=1.289803717623962e-05, P172=1.750544047593357e-05, P173=3.201540713326032e-05, P174=5.8334570751304184e-05, P175=6.473052398654704e-05, P176=4.537390022962997e-05, P177=3.9943194299825436e-05, P178=1.8851475764288344e-05, P179=2.880387636701151e-06),Row( P0=-696.2185540405177, P1=60.14570382685148, P2=-12.640754084579376, P3=7.040645065992057, P4=-3.8533635786782705, P5=-9.423219205445335, P6=-2.8350282418010284, P7=2.6369393819489515, P8=-0.216778472823019, P9=-0.23931233477385497, P10=-7.947278720933333, P11=1.2818523475702273, P12=1.7690737911294223, P13=-1.610691808789723, P14=5.189491608086925, P15=-2.1417716692438034, P16=1.52427480970624, P17=5.514344310120139, P18=-3.514211436639119, P19=4.386769209517574, P20=-3.1277490362288143, P21=0.9738004955215389, P22=4.154458300621915, P23=-0.32187285961699336, P24=2.7850849144107848, P25=3.3675200135765007, P26=0.7763214754642569, P27=1.693006519052649, P28=-1.802637085938032, P29=0.05257000996353537, P30=1.9846559795838614, P31=3.095164028035826, P32=6.5245811553274615, P33=6.6225081429839685, P34=4.4238296872247656, P35=6.193656950273107, P36=1.7651769157304114, P37=4.397256655471305, P38=3.4480318032320554, P39=1.2665911243546901, P40=0.7450591766202407, P41=0.6447492122219616, P42=0.6688203560643611, P43=0.7225022769782002, P44=0.7591988126819799, P45=0.7697213581531291, P46=0.6853022232793313, P47=0.6141707713142616, P48=0.6321956340319717, P49=0.6967196143340653, P50=0.7292096335117799, P51=0.7441834501268852, P52=0.00018152613231770575, P53=4.016905626303388e-05, P54=3.3505498727810783e-06, P55=4.939482856581526e-06, P56=1.3568254951754191e-05, P57=0.00027637704246806364, P58=0.0027141116808186605, P59=0.005046575779383895, P60=0.0032182800624395167, P61=0.008419376683910254, P62=0.012493208793901847, P63=0.006297366781989388, P64=0.0030843055977752293, P65=0.006310651795722286, P66=0.017934687492660164, P67=0.031052021515767452, P68=0.016245123592463807, P69=0.004263319932264325, P70=0.005277153929250829, P71=0.005872639703370881, P72=0.015175325506687272, P73=0.026183770566814136, P74=0.014111966382415124, P75=0.005485574734984943, P76=0.004818788432657649, P77=0.0034244205543705305, P78=0.0033708315843507986, P79=0.00477634262364719, P80=0.005024457608592247, P81=0.007615166560073892, P82=0.009913159626261776, P83=0.014830883449538368, P84=0.03662939501151822, P85=0.037571997098165444, P86=0.005100107862330028, P87=0.00958984205111522, P88=0.03043133145157377, P89=0.01951878453172523, P90=0.01532474931337092, P91=0.016885755276054702, P92=0.01737844805908395, P93=0.023928140501059597, P94=0.030117067594010095, P95=0.006641707931829456, P96=0.0019829927077393635, P97=0.012201801411791403, P98=0.022240813236269495, P99=0.012187183423545519, P100=0.014098766678824317, P101=0.008680047168134641, P102=0.0006905206857952091, P103=0.00035550397725638146, P104=0.000303179579472782, P105=0.0012384595875287354, P106=0.0022468729808177617, P107=0.0006927356318464596, P108=0.0004965134303983103, P109=0.0006361062391508103, P110=0.004020033718976282, P111=0.0083475820963592, P112=0.0042083332095201875, P113=0.0023520458635669372, P114=0.0015754881703964117, P115=0.0019236740516640892, P116=0.0008359195940048509, P117=0.001350815861483815, P118=0.0020483351207632758, P119=0.000320079202465811, P120=0.0018652082381448362, P121=0.0012346700461852143, P122=0.00044728335505016243, P123=0.0006400708302547335, P124=0.0011657149000661223, P125=0.0003527659574448674, P126=0.0006406787182271856, P127=0.0005050412379467742, P128=0.00028639247487197183, P129=0.0003481761436518014, P130=0.00020472595136172586, P131=0.00023788740120828634, P132=0.00014229323890525242, P133=0.00019031379367925167, P134=0.00012010759634860862, P135=7.512092027016229e-05, P136=6.455857996394796e-05, P137=6.717314481315648e-05, P138=4.3527599066707514e-05, P139=6.026430445086364e-05, P140=5.8188664164939936e-05, P141=9.966136761436283e-05, P142=7.493767122448781e-05, P143=9.132679063779239e-05, P144=0.00016285107029400406, P145=0.00029527435122984305, P146=0.00030714866463671094, P147=8.776322369599754e-05, P148=1.9081243701682036e-05, P149=2.1049458578337788e-06, P150=9.214485393091254e-07, P151=1.4783686998209485e-06, P152=2.035754794666805e-06, P153=4.6231605190579604e-06, P154=5.644913202223692e-06, P155=6.120578225470576e-06, P156=6.4841644934639025e-06, P157=1.0809576565076414e-05, P158=1.1341332093398015e-05, P159=1.3558187027449063e-05, P160=7.933936227574455e-06, P161=4.756667034891216e-06, P162=3.7542952333281712e-06, P163=3.330534407416961e-06, P164=3.15869681179881e-06, P165=3.251356680361825e-06, P166=1.8084326879794911e-06, P167=1.6367932847987564e-06, P168=2.1642615303628543e-06, P169=3.182481422378505e-06, P170=2.7764289210608424e-06, P171=1.6178095081724519e-06, P172=1.3125543988097022e-06, P173=1.0128267166612634e-06, P174=8.144293092678508e-07, P175=2.1161411946065748e-06, P176=2.4616358817440624e-06, P177=1.6596857265219557e-06, P178=7.841246244759177e-07, P179=7.171400288768523e-08),Row( P0=-737.2737880620061, P1=73.00071488585299, P2=1.5541409038242817, P3=7.053385664826288, P4=-0.6303042862004915, P5=-3.4837371388014384, P6=1.5175049344558227, P7=5.896960154167887, P8=2.8475211673868364, P9=-0.7802101796275323, P10=-2.7250174506286435, P11=-0.615391664260249, P12=-1.5131574218589687, P13=-2.750957798884234, P14=1.365594344477037, P15=-2.6539234854668994, P16=0.9021047010856782, P17=0.8966247695239165, P18=-0.35689321647751976, P19=1.456098730722244, P20=-5.22219237135041, P21=0.44444467593245013, P22=-0.5319140816318464, P23=0.21501841661416038, P24=-3.2959336818035263, P25=-2.3036356111854244, P26=-1.8806873692610562, P27=-2.7490946012093933, P28=-2.6985799655883635, P29=-1.090870289577947, P30=-2.5614769841091123, P31=0.6073112323847832, P32=0.18123530600531948, P33=2.029038813448486, P34=2.6282830548781537, P35=3.661252239853478, P36=2.945528949672698, P37=4.706493710622012, P38=2.8433470143960786, P39=4.563659521693476, P40=0.7417137150426057, P41=0.711341073704396, P42=0.7317937053552555, P43=0.731101220853288, P44=0.7515899717458892, P45=0.776447725863113, P46=0.7401710809553591, P47=0.7607357334061656, P48=0.7746301371144567, P49=0.7467747684367945, P50=0.7580738550336367, P51=0.7748321832655101, P52=2.271618408336816e-05, P53=9.629866156581338e-06, P54=5.409622339031823e-06, P55=5.3124108432337824e-05, P56=0.0011554185966876002, P57=0.009692507419561194, P58=0.009462606765035457, P59=0.0048981712042317285, P60=0.0012228049293676207, P61=0.0004562414222496936, P62=0.000983066204242104, P63=0.002895110825926607, P64=0.00552555404445935, P65=0.00914006131966012, P66=0.010724732707720163, P67=0.011908284801208587, P68=0.005064675502895526, P69=0.0010250747618910142, P70=0.0018308115038481562, P71=0.0022071136924552326, P72=0.0021975806123579714, P73=0.0031788267935826996, P74=0.0023871422632841277, P75=0.0026766615370492366, P76=0.0031364946247024053, P77=0.003879977695414425, P78=0.0040869287166997535, P79=0.003786233076468015, P80=0.005393279094587077, P81=0.004167218649738934, P82=0.003125754847299958, P83=0.005541771489880423, P84=0.007661283231177364, P85=0.004388295572055485, P86=0.0027626914381796196, P87=0.00459983957781607, P88=0.008489301119107432, P89=0.003555128818534948, P90=0.001045600148320306, P91=0.0018858644041898573, P92=0.010368687929891816, P93=0.009455959630143207, P94=0.007957417929043146, P95=0.0011917635052108957, P96=0.0017089428129011361, P97=0.004728072007849663, P98=0.002797770282999129, P99=0.0011541075534423199, P100=0.00021609378857492077, P101=0.00020026049980521974, P102=0.0001877039161817534, P103=0.0002582159963638113, P104=0.00019479729923842026, P105=0.0003503584731352602, P106=0.0006703321781706706, P107=0.0007276925697173657, P108=0.0002498861144009613, P109=0.0003381294777719644, P110=0.001830052355583798, P111=0.0010664763053925882, P112=0.0002743721701021815, P113=0.00028699332258613145, P114=0.00010000509961330949, P115=0.00013566865723712695, P116=8.144378673244877e-05, P117=0.0002957928991949325, P118=0.00012215193681290457, P119=8.01735785089411e-05, P120=0.00015150597185892864, P121=0.0001823332549341216, P122=0.00020239714060579114, P123=0.00034925821355256026, P124=0.00013376073824270383, P125=0.00014598102482380978, P126=0.0001242015336801525, P127=0.00017942668655162593, P128=6.880520686229425e-05, P129=9.631146398111847e-05, P130=6.769024604561983e-05, P131=8.529094803589888e-05, P132=6.894465000881985e-05, P133=4.3372412229212704e-05, P134=3.642917093521742e-05, P135=6.689447088066328e-05, P136=0.00010136375993774772, P137=4.497094830545937e-05, P138=2.0150374595979156e-05, P139=1.2642559650395493e-05, P140=1.5101271883234166e-05, P141=1.6278139151662414e-05, P142=1.9803966090038473e-05, P143=2.9963114689561976e-05, P144=2.691273028856647e-05, P145=2.490864733211637e-05, P146=1.761149960647058e-05, P147=1.4798697498939135e-05, P148=1.5110294286853557e-05, P149=2.454225403229559e-05, P150=8.208485039840256e-06, P151=5.683029907255718e-06, P152=4.961454036671474e-06, P153=1.2996089830092585e-05, P154=1.0603754949838763e-05, P155=2.1638462546422976e-05, P156=4.4591511245379805e-05, P157=6.28938454847992e-05, P158=8.503969739972502e-05, P159=0.00010631990696131796, P160=8.472535955821782e-05, P161=6.915419148656687e-05, P162=5.9467020452082226e-05, P163=3.162841820179304e-05, P164=1.680089361672358e-05, P165=1.26587598087291e-05, P166=1.3454861426095573e-05, P167=2.22644892583662e-05, P168=2.4228487798396923e-05, P169=1.6599057417923228e-05, P170=1.1963268492485312e-05, P171=7.0061503831382485e-06, P172=4.661740520686713e-06, P173=1.2225726947432095e-05, P174=2.6862453317602194e-05, P175=3.53501366174269e-05, P176=3.8448517232790894e-05, P177=4.153291839242394e-05, P178=1.6874623978522468e-05, P179=1.5557919386060158e-06),Row( P0=-664.4963508811742, P1=54.64770857702074, P2=-1.4665630379736092, P3=5.3485708025479335, P4=-4.683482642299128, P5=-6.98215456395477, P6=-3.7188437207551504, P7=1.81369898898478, P8=-1.959408811186553, P9=1.516316892058121, P10=-4.0718811869276275, P11=0.588424282564101, P12=-1.671419750162079, P13=-3.0786579961650524, P14=2.0619258731892405, P15=-2.00051820894273, P16=0.9737024203919721, P17=1.3737041919033064, P18=-0.1444991563339494, P19=3.0018745978745836, P20=-4.4315178055260445, P21=-0.07105790664743497, P22=2.3879427172780443, P23=1.6078028666004462, P24=1.4604907622908159, P25=5.317134927088896, P26=5.465374164659768, P27=6.73929387757767, P28=6.288745307625207, P29=6.54640342798329, P30=3.4580767317554177, P31=5.397143856297201, P32=1.6744670498682597, P33=2.8795813460701423, P34=1.226992462480572, P35=1.6501912550591848, P36=0.4568626054694024, P37=1.5921303499061499, P38=1.3268775039938177, P39=1.0246407908341992, P40=0.7370761723060676, P41=0.7394742888129789, P42=0.7927036351041414, P43=0.7963510955421536, P44=0.77172628720444, P45=0.7321021556115037, P46=0.6964601092637533, P47=0.682759220104013, P48=0.6978817895449886, P49=0.7274203929423648, P50=0.727630576479962, P51=0.7455736567102575, P52=0.0002141438829103939, P53=8.296003893702401e-05, P54=2.754138587423387e-05, P55=3.067681460394364e-05, P56=0.00011054197454002726, P57=0.0007703226851966674, P58=0.001035889822413484, P59=0.00615306770944388, P60=0.026771124499953964, P61=0.03418985429302191, P62=0.019495931816245628, P63=0.00864795005171561, P64=0.0027708602011179977, P65=0.0009950046901584777, P66=0.0023435951346404017, P67=0.005121411596055022, P68=0.01079877815060675, P69=0.03746238297306561, P70=0.07816485069142752, P71=0.04210888462268465, P72=0.014078631832429625, P73=0.009216331188560833, P74=0.0030233744263450267, P75=0.0025568126293397304, P76=0.0038077802427835066, P77=0.012078231916020673, P78=0.025026761959655214, P79=0.02384751765690004, P80=0.027186625264272454, P81=0.07056408491138096, P82=0.09387813440087425, P83=0.06313318658172404, P84=0.03974796022174494, P85=0.013744187625818345, P86=0.019185345900105795, P87=0.023488090401961845, P88=0.014475665696612246, P89=0.005981854789571611, P90=0.008563739450650752, P91=0.009285317622983864, P92=0.01415718008034757, P93=0.012831648126236946, P94=0.014523675086929122, P95=0.012127494198288064, P96=0.018761628657989634, P97=0.0076817896392489355, P98=0.005505459182083322, P99=0.006561478604963043, P100=0.003694368103003626, P101=0.002288616951300819, P102=0.001412096166153346, P103=0.0008616709227936752, P104=0.0011608735669128422, P105=0.001518576736296555, P106=0.0016159442170977948, P107=0.00227591711377178, P108=0.002151781507218842, P109=0.0015238731515509196, P110=0.0021080421120005907, P111=0.002775097612631912, P112=0.002106550634039929, P113=0.0013245848776469683, P114=0.00044524621277599875, P115=0.00037458309355537753, P116=0.0005919735261115082, P117=0.00038983382473431894, P118=0.0006492744251320942, P119=0.0004039814240239967, P120=0.0008624738629128113, P121=0.0010915087577163375, P122=0.0006286308526955241, P123=0.001701157829225151, P124=0.00042664609502836786, P125=0.000573369560726931, P126=0.0003080601447431827, P127=0.00033320811517869274, P128=0.00040619104860821915, P129=0.00027524291842624644, P130=0.0003384907133897244, P131=0.00011437570504003792, P132=0.0002599204682075269, P133=0.00034599142316662856, P134=0.0003323706122082169, P135=0.00021545216356371206, P136=9.599345378780058e-05, P137=4.956612517271492e-05, P138=7.671110299100501e-05, P139=6.149575996570256e-05, P140=4.330913155089544e-05, P141=4.60994105810839e-05, P142=4.997732466349568e-05, P143=9.00995966714264e-05, P144=0.00016341548725473895, P145=0.00015637295488031447, P146=8.87153922617105e-05, P147=6.417982931175457e-05, P148=5.093330852203258e-05, P149=3.512566775319239e-05, P150=2.5764607727154935e-05, P151=3.0060156288824378e-05, P152=3.158294981590002e-05, P153=3.747694659376441e-05, P154=2.557227338564784e-05, P155=3.87797699747318e-05, P156=7.604913837559744e-05, P157=0.00011446901767427267, P158=0.00011424920732551077, P159=0.00011441625277460256, P160=9.953415682704346e-05, P161=7.718373075155932e-05, P162=6.430752820381656e-05, P163=3.346124695874665e-05, P164=3.067082085195686e-05, P165=4.876728197490802e-05, P166=6.512596933678239e-05, P167=7.490948078948064e-05, P168=5.4393056626653235e-05, P169=3.996850047221859e-05, P170=2.7532037226624178e-05, P171=1.7602062257582687e-05, P172=1.2439027503186178e-05, P173=3.569360827407803e-05, P174=4.655744153918163e-05, P175=5.961281546243654e-05, P176=5.653572039484519e-05, P177=3.9604522374729744e-05, P178=1.6192345928205897e-05, P179=2.609748038730739e-06),Row( P0=-662.5637620112615, P1=53.71354767329698, P2=-11.600696595379423, P3=2.3319072074194223, P4=-11.013785259680192, P5=-9.324841100551687, P6=-10.233752811368023, P7=4.567478148706362, P8=-2.132391911202621, P9=-3.4095583293850797, P10=-6.952401368345619, P11=-1.7871067036501003, P12=1.5327659149716382, P13=-4.037846217796705, P14=4.52292855844138, P15=-2.6477234533603515, P16=3.5521422447373143, P17=5.829391809743049, P18=0.49171454044037266, P19=8.810877955102708, P20=-1.4272298862983757, P21=3.4920903094422258, P22=3.739296197149483, P23=0.7276904425387685, P24=3.0338477817690497, P25=2.3414816573200383, P26=2.4806794603400433, P27=4.260615374762837, P28=0.8583593465103986, P29=2.606167432767543, P30=0.7838803499818979, P31=2.7537783725585006, P32=2.161503937755664, P33=2.5818358896461304, P34=2.6269817943427847, P35=2.7338751049733254, P36=1.2560716116756765, P37=2.3423112158681145, P38=0.8422701839848878, P39=0.07769938370275274, P40=0.7411505672020045, P41=0.6562453087955991, P42=0.6897561758330254, P43=0.7037270939077243, P44=0.7094435844461808, P45=0.7256490083616985, P46=0.7048835310288596, P47=0.6546657737351537, P48=0.6763384150290623, P49=0.7001664181125088, P50=0.7117374082258996, P51=0.7683465080614864, P52=7.051637877832603e-05, P53=1.68006088493664e-05, P54=4.63639731669401e-06, P55=1.5193062942794838e-05, P56=5.836747384390477e-05, P57=0.00035241138604801746, P58=0.0014472870480349367, P59=0.0027025919253905145, P60=0.004674321345352011, P61=0.011524061413769916, P62=0.017855795562629687, P63=0.02139983351093876, P64=0.010225624022732784, P65=0.00957247762119793, P66=0.017365653898556148, P67=0.025423161701142685, P68=0.010429528617297358, P69=0.016293000269527267, P70=0.023630068363909996, P71=0.010980957316514271, P72=0.007272047554558027, P73=0.008882873527170767, P74=0.007570950191402598, P75=0.00622727337892128, P76=0.006145595988097575, P77=0.009404564646837074, P78=0.023123770429984362, P79=0.04175851943116765, P80=0.02573353530095781, P81=0.02031529211364707, P82=0.04213963306126083, P83=0.046596530546537424, P84=0.03431002454744231, P85=0.021365201875467043, P86=0.011866169966919627, P87=0.03397093042007715, P88=0.07480682513916859, P89=0.03654765636532849, P90=0.03910512995471887, P91=0.03526981318970408, P92=0.04401442385996527, P93=0.038561624068759896, P94=0.0739259174041733, P95=0.03350191746332503, P96=0.014763012582104126, P97=0.024432542952740215, P98=0.01972898919668327, P99=0.009339450367069742, P100=0.003126386002810727, P101=0.00101151705380242, P102=0.0006365877861390124, P103=0.0012239537396303254, P104=0.0011259013762327382, P105=0.0012809161901758492, P106=0.0020565392722403757, P107=0.0029802853829504496, P108=0.0017200924173909893, P109=0.0015813180609665402, P110=0.008185358272360835, P111=0.009210224059365465, P112=0.005827958211658771, P113=0.0023077049913434056, P114=0.0017288553912930425, P115=0.003248659970728945, P116=0.0032789996320004527, P117=0.0015876379365303602, P118=0.003147523944437146, P119=0.002605362982610068, P120=0.0028312001770085596, P121=0.0022519770790486427, P122=0.0015212339223152002, P123=0.0010964946667895354, P124=0.0006287998967457469, P125=0.0005292812248990308, P126=0.0009666861916548605, P127=0.0005711737823819784, P128=0.0008688279338900116, P129=0.000674377592784779, P130=0.000493371159505641, P131=0.000331670778417116, P132=0.00035774312021449574, P133=0.00042238666158749943, P134=0.0002921230503305092, P135=0.00014907640334272567, P136=0.0001275343127895156, P137=0.0001188648844827288, P138=5.947787532939694e-05, P139=3.83684179366676e-05, P140=4.385966872209796e-05, P141=6.245682972112422e-05, P142=6.220997849629677e-05, P143=0.00010002184507111135, P144=0.00017887619452615437, P145=0.0004476154705838519, P146=0.0004569130124686258, P147=0.0003146809183355889, P148=0.00010325651336549882, P149=2.8937289393396377e-05, P150=1.0673720402147236e-05, P151=6.403231345223384e-06, P152=6.140761118076082e-06, P153=1.882229668493052e-05, P154=3.2213541697380946e-05, P155=3.8156631139627504e-05, P156=7.099667758838844e-05, P157=8.908258551370188e-05, P158=0.00012088236811078694, P159=0.0001815769887300448, P160=0.00017807603747204025, P161=8.37132815914319e-05, P162=5.412487032019373e-05, P163=4.2643093269583095e-05, P164=4.428147610122573e-05, P165=2.9431428589463236e-05, P166=2.989434426253342e-05, P167=3.198993960777839e-05, P168=2.87139128770009e-05, P169=1.3836017664098887e-05, P170=1.0989299630250723e-05, P171=1.0767780123004941e-05, P172=8.014611840753466e-06, P173=1.3342195600561154e-05, P174=1.0154846769882995e-05, P175=1.0230448643204688e-05, P176=7.042044322606848e-06, P177=1.125688086896001e-05, P178=8.400215805436701e-06, P179=7.299466391071909e-07),Row( P0=-527.4917703999133, P1=28.24983333992073, P2=-14.666816829028416, P3=-5.153842024553248, P4=-12.988354444053366, P5=-9.445433527112664, P6=-6.174522653375113, P7=-0.14151268898415964, P8=-5.8974372392081715, P9=1.6039233273430207, P10=3.55553631248576, P11=5.046883568671008, P12=8.858849744864244, P13=5.839636362735243, P14=7.663223503235869, P15=5.745673210472794, P16=2.6888303845323147, P17=5.78501790458022, P18=0.6488107971755334, P19=3.665090408383755, P20=1.067836144762273, P21=2.528184334945002, P22=4.094042651337108, P23=2.9352226174572253, P24=2.5585455380498665, P25=4.501979855903481, P26=3.6786480290910206, P27=2.4792786131332094, P28=2.3505345929975068, P29=3.1553168244136747, P30=0.13025197286965515, P31=4.727662802239277, P32=1.1800502073009067, P33=2.551173791390686, P34=0.7230136236825938, P35=2.1521611622885946, P36=2.2243619477513645, P37=4.191775794322198, P38=1.1667818490179716, P39=1.2495762602090315, P40=0.6548894093151736, P41=0.7012367815472608, P42=0.7149295756440914, P43=0.6935160495986672, P44=0.7370053174067781, P45=0.7422025082416449, P46=0.6891241493474779, P47=0.6421093613643235, P48=0.6511665069801073, P49=0.678883992056024, P50=0.699909262212943, P51=0.6823898385099678, P52=0.07937915763943024, P53=0.009101658555342507, P54=0.0003139975291828386, P55=0.00013267135941678047, P56=0.00014338438106618054, P57=0.00023051085587342046, P58=0.00018918972569756826, P59=0.0003318307657597077, P60=0.001365461330143594, P61=0.003799761758927316, P62=0.008803898392447482, P63=0.01326652614887391, P64=0.009268771481725542, P65=0.023135401713404145, P66=0.0661859713774298, P67=0.1126313475457331, P68=0.2391828810601781, P69=0.6016668540418347, P70=0.614943683590477, P71=0.3782012326049633, P72=0.5842798031666016, P73=0.7364556800561972, P74=0.44597412624160415, P75=0.14080021584321523, P76=0.07623829764082675, P77=0.1356942832382735, P78=0.32143804043707547, P79=0.3450669633619886, P80=0.16034641518982676, P81=0.12346868981601035, P82=0.12158562134362182, P83=0.09685688599211653, P84=0.14012746040096694, P85=0.35095074846201146, P86=0.5344242335159788, P87=0.6360163779613255, P88=0.4938283947138533, P89=0.3165388562800991, P90=0.3510628331375569, P91=0.613757629173237, P92=1.2986531760678193, P93=1.426077181772028, P94=0.766475588459111, P95=0.30023461613723207, P96=0.21144617128393225, P97=0.1664953968785709, P98=0.1856876561003009, P99=0.3184517815421487, P100=0.2639999027044631, P101=0.2774093105247044, P102=0.41249086398495216, P103=0.7218885632129368, P104=0.0865502124338715, P105=0.062130975524808264, P106=0.1075826448753938, P107=0.1023072188884848, P108=0.05448245967210757, P109=0.017931843304116478, P110=0.015975323370042323, P111=0.033604481939393006, P112=0.050193811349282844, P113=0.010268071899399352, P114=0.004627235402651759, P115=0.014864312195340398, P116=0.01309619886096292, P117=0.007284898854126843, P118=0.015965310860964567, P119=0.006200205225174975, P120=0.015624311507706323, P121=0.029608346534779565, P122=0.008225288730853316, P123=0.005756775728274609, P124=0.014185569358507942, P125=0.043300445301701206, P126=0.01090739115976191, P127=0.004293937733638994, P128=0.005096048182159829, P129=0.0086157929888289, P130=0.009389302888392622, P131=0.007143379987539507, P132=0.006755120080359035, P133=0.011261106340431301, P134=0.00832060401437753, P135=0.004088233987189643, P136=0.0038775702592066677, P137=0.0012868915647496692, P138=0.0017559691001039393, P139=0.0025454076889456713, P140=0.003203518315351321, P141=0.0045206248804695115, P142=0.001821492417875159, P143=0.0015474715338371489, P144=0.0016811306475365188, P145=0.0012044122092792083, P146=0.0010113490690247782, P147=0.00047608211424442914, P148=0.0005052631717230196, P149=0.0006300881773374262, P150=0.0005283945379860658, P151=0.0007310737323148427, P152=0.0005479979840445058, P153=0.00048373116670615614, P154=0.0006415156706861576, P155=0.0007670795889218996, P156=0.0017702405510577985, P157=0.002267358326148169, P158=0.0015499704062771225, P159=0.001628210380054984, P160=0.0015926514007330423, P161=0.0010493392695805409, P162=0.0005319638588134093, P163=0.00029801107319811656, P164=0.00035758371059794525, P165=0.0002897342208173547, P166=0.0002517431932995538, P167=0.0002208047175689823, P168=0.00025660369104796066, P169=0.00023992437309779244, P170=0.00023722046637103026, P171=0.00015134559728919905, P172=0.00011419854092804971, P173=0.00014556611210385416, P174=0.00017884197808672856, P175=0.0001858886815943087, P176=0.0002210385100564281, P177=0.0002560039308749961, P178=0.00011768904338453837, P179=1.7641056046642268e-05),Row( P0=-731.6506346000733, P1=71.1273729272312, P2=-7.866437155593661, P3=6.7796372980901305, P4=0.6346626640347656, P5=-7.891380729834898, P6=-2.4975508311145624, P7=2.334776385737645, P8=2.4766915960280222, P9=-1.4626898987992778, P10=-7.716984169866268, P11=-1.5107745718961805, P12=0.33745419009738625, P13=-2.799753444734305, P14=2.180182393362356, P15=-2.9117103409622924, P16=0.4473446098978607, P17=2.2265596225859694, P18=-2.7842094561890125, P19=2.7683583219646644, P20=-2.1522809883244727, P21=-1.4847400569939777, P22=-0.4488391193782958, P23=-1.5396508907995572, P24=-3.5612289104858443, P25=-0.3315811895707268, P26=-2.0670167126416303, P27=2.236889877440235, P28=1.2356477759818494, P29=0.216842146418673, P30=1.2812533857324915, P31=2.3589389240615937, P32=1.2646915279549569, P33=1.848373246888144, P34=1.0984627309857438, P35=1.4796267278776536, P36=0.7972305907866739, P37=2.9231340506586463, P38=5.191062799406661, P39=6.575977510188664, P40=0.7302719775336537, P41=0.6855146451037062, P42=0.7265066234666366, P43=0.7751010287861559, P44=0.7119698039252568, P45=0.6820463714022909, P46=0.6868916594727875, P47=0.7309058685805712, P48=0.7840326457028062, P49=0.7273007939791891, P50=0.6955027103471662, P51=0.7106416270094571, P52=2.9217169352051942e-05, P53=8.676269803918213e-06, P54=2.6886733016196196e-06, P55=9.289942067142749e-06, P56=0.0001592835389994409, P57=0.002729193162826215, P58=0.004012454662409238, P59=0.003572444964364342, P60=0.005515259532021035, P61=0.003706606689814194, P62=0.0010230053419788376, P63=0.0009063854961072818, P64=0.003966888737146509, P65=0.005566259534209998, P66=0.0026406690865631703, P67=0.0024564875209268943, P68=0.003899871553010567, P69=0.005752327338029876, P70=0.0060252178999039676, P71=0.004738538859571175, P72=0.0022257880734341193, P73=0.001108837637719354, P74=0.0005705398434436647, P75=0.0008784566249855416, P76=0.0025230141398485165, P77=0.014534788167870738, P78=0.02099065434545125, P79=0.011032428004381513, P80=0.012661391662300835, P81=0.009157445739932558, P82=0.0020254217865841085, P83=0.0019903739511226834, P84=0.009358795881151061, P85=0.010645234975639534, P86=0.003607450925644912, P87=0.002482471520628415, P88=0.001968807690051803, P89=0.00491438117001825, P90=0.01365426575837122, P91=0.015175778368458492, P92=0.011318562299293447, P93=0.0036104918914895063, P94=0.0019069813173852631, P95=0.004042930068963045, P96=0.013972835169161889, P97=0.013693951965479243, P98=0.0014273227087852473, P99=0.00046624369032591544, P100=0.0002859030716496227, P101=0.00040801322571979364, P102=0.00021874462148586965, P103=8.316793587764618e-05, P104=7.971419839630691e-05, P105=0.0013504465206816865, P106=0.0009598022251981621, P107=0.00014927223444689112, P108=0.0006300218525025497, P109=0.0012807730659676233, P110=0.0005537530384492332, P111=0.001629485224240431, P112=0.0013668365454647872, P113=0.0005640266532682355, P114=0.0004902799200024254, P115=0.0008706623700038445, P116=0.0010355965707179741, P117=0.0005250519114270934, P118=0.0006051449996442025, P119=0.0002621700651756648, P120=0.0004419754485065894, P121=0.0001363626841284957, P122=0.00017602740718451626, P123=0.00018307994793833107, P124=7.985148088611735e-05, P125=0.0002240274027248816, P126=0.0001025576177233551, P127=0.00010405962911008842, P128=6.395777164072352e-05, P129=7.94548106747535e-05, P130=7.743785361057893e-05, P131=3.584803798781541e-05, P132=9.163952903686203e-05, P133=6.65669142102321e-05, P134=9.565960832775342e-05, P135=7.755135120494709e-05, P136=3.817961611707026e-05, P137=1.3090844966647778e-05, P138=1.3997482543639205e-05, P139=1.0606313613008284e-05, P140=1.8560843187734435e-05, P141=2.151676024653228e-05, P142=2.1866294329892477e-05, P143=3.998976201498038e-05, P144=5.998900208850746e-05, P145=8.698679760290271e-05, P146=3.252431918459064e-05, P147=6.9235945465510445e-06, P148=2.410938231672401e-06, P149=1.7635970846020866e-06, P150=3.4794355344692724e-06, P151=4.807016866969669e-06, P152=4.647204636422813e-06, P153=3.258834000656908e-06, P154=2.6155898151930947e-06, P155=3.0289317699086777e-06, P156=2.2504241341940097e-06, P157=1.5279056716149111e-06, P158=1.7397910146546852e-06, P159=3.533520178259732e-06, P160=2.036341437941836e-06, P161=1.2109417609708368e-06, P162=1.7516081117480048e-06, P163=1.1488126522348565e-06, P164=1.0852661430692101e-06, P165=1.0666402406383737e-06, P166=7.373894671230132e-07, P167=7.741491268690514e-07, P168=5.61294274786195e-07, P169=4.671980431784285e-07, P170=3.1983989903454427e-07, P171=4.125517535967682e-07, P172=2.478032392414743e-07, P173=2.409355502001203e-07, P174=3.99597529995591e-07, P175=5.20071263785658e-07, P176=7.171592876839513e-07, P177=6.801781144480568e-07, P178=2.9242246917735184e-07, P179=3.242766849582692e-08),Row( P0=-743.1980301467364, P1=67.7937606917119, P2=0.0987344268052917, P3=6.2259476888596215, P4=-3.335050197931124, P5=-5.172846622471142, P6=-0.31954680677353736, P7=4.106488707205984, P8=7.3094583189348565, P9=-0.06898290008971324, P10=-3.2808232516649487, P11=1.4182209006868336, P12=-1.7717792604696356, P13=-1.5018057622630863, P14=3.728938387575384, P15=-1.6278415163112103, P16=1.3968789119027707, P17=5.421008927582093, P18=-4.373618345686083, P19=4.685095941540167, P20=-4.439735202935609, P21=-1.37578847610743, P22=0.3270812031215024, P23=-3.84670073860699, P24=-2.7112074539942683, P25=-0.672729479739954, P26=-1.1484220293228087, P27=-0.06476775533118592, P28=1.3083978154690183, P29=2.9491786806928597, P30=2.0872927821014238, P31=3.847862837835852, P32=2.1682929837835125, P33=2.1327855003088048, P34=2.8770045237987905, P35=3.1358007578582927, P36=2.028733709207631, P37=4.3182027786660715, P38=4.043504810823709, P39=2.4367522913592445, P40=0.8045452091577777, P41=0.7570988507003354, P42=0.724857100607665, P43=0.7373852266816444, P44=0.7202890105586282, P45=0.6968329203749658, P46=0.6987264502042277, P47=0.6775686998578396, P48=0.7394082092098383, P49=0.7625175500034096, P50=0.7690879564898282, P51=0.8120453053746338, P52=1.830689840082653e-05, P53=7.649092498723639e-06, P54=7.261360329413035e-06, P55=4.025399870764393e-05, P56=0.000134619331598727, P57=0.0011178879002981498, P58=0.002806647303458126, P59=0.002415015106357037, P60=0.0022116324688957745, P61=0.001401412321964742, P62=0.000476403760503884, P63=0.0002564152615248869, P64=0.0008156969443561399, P65=0.0028750069506225857, P66=0.003850617541324513, P67=0.002797684518650302, P68=0.0018110495772453872, P69=0.0025304236179106284, P70=0.00396194824493493, P71=0.0019027896817067297, P72=0.001227984487307844, P73=0.00174082037960271, P74=0.0014386107147621838, P75=0.0011877750265853893, P76=0.0013013665031109318, P77=0.0032495715673512417, P78=0.006473942076183733, P79=0.0084367383684951, P80=0.006243153317316599, P81=0.002565642717048435, P82=0.0028593819221378637, P83=0.002953878675354813, P84=0.005032145334778861, P85=0.010452510704039675, P86=0.009681574471375722, P87=0.009412017921666939, P88=0.016033237665830938, P89=0.004962167720065126, P90=0.0034609273314646495, P91=0.0031246051761426217, P92=0.0024275662095844955, P93=0.0009960528146916016, P94=0.005577029555649768, P95=0.013235455325192116, P96=0.0027575329458106707, P97=0.00012375473243065278, P98=7.498951651285397e-05, P99=0.00025723405381154795, P100=0.0007798004197890115, P101=0.0008616910880616357, P102=7.983254985419922e-05, P103=5.75805096699838e-05, P104=8.89465204856896e-05, P105=0.00036117332805764784, P106=0.00035840304462121897, P107=8.283303765113064e-05, P108=0.00011016399352860262, P109=0.00035643678690414467, P110=0.0007682036789425887, P111=0.0008227123168028753, P112=0.0012934825493029405, P113=0.000666174442262742, P114=0.0002794075452676483, P115=0.00017130577239855144, P116=0.00028928536887632333, P117=0.00022737730482968037, P118=0.0001864976035360026, P119=0.00012033320993887965, P120=9.267076005368887e-05, P121=8.571940426389571e-05, P122=8.938861756921126e-05, P123=8.115223824775089e-05, P124=6.17557312330609e-05, P125=8.970112652417291e-05, P126=6.404282960322744e-05, P127=8.143925303559758e-05, P128=2.9005000322790556e-05, P129=2.7047869021656438e-05, P130=4.068460912312244e-05, P131=6.470841002058167e-05, P132=8.772241773459794e-05, P133=0.00014252065780632448, P134=4.7454171138051555e-05, P135=1.6961079537940127e-05, P136=9.708386383009125e-06, P137=4.7653455288902205e-06, P138=3.809204837018175e-06, P139=6.690481755674265e-06, P140=8.412845190574119e-06, P141=1.4925492430461406e-05, P142=2.2605808915465574e-05, P143=5.4751830414941724e-05, P144=9.535643730814735e-05, P145=0.00011606919497346739, P146=9.206209559864173e-05, P147=1.9120236287051076e-05, P148=6.050022932782314e-06, P149=3.4121491123614003e-06, P150=2.407092884183399e-06, P151=1.3216399383968868e-06, P152=1.6966241084950795e-06, P153=2.8460039002520606e-06, P154=4.544672755234306e-06, P155=6.306004567359578e-06, P156=8.267281651796768e-06, P157=1.0862407634743572e-05, P158=1.545791193557348e-05, P159=1.6397806713435506e-05, P160=8.952757586253752e-06, P161=5.098007902510591e-06, P162=3.872140480007941e-06, P163=2.711694022216837e-06, P164=2.31009259835484e-06, P165=3.834328937392137e-06, P166=4.365147764742687e-06, P167=5.770913193745023e-06, P168=4.728097917496966e-06, P169=4.06906839064e-06, P170=3.370841297230501e-06, P171=4.5210484575890414e-06, P172=3.6282872650567477e-06, P173=3.6416732385892973e-06, P174=5.593431246149425e-06, P175=5.652359045499131e-06, P176=5.808677939970991e-06, P177=6.540148497503414e-06, P178=2.8382273188353816e-06, P179=2.342440860875466e-07),Row( P0=-688.303424375093, P1=55.796354043379935, P2=0.7818345893558792, P3=3.124355460879492, P4=-4.567781371842557, P5=-3.8329576467912614, P6=-1.6522841256373484, P7=3.2819373778671426, P8=-1.3340613498995015, P9=1.2588903787091656, P10=-1.9716414842778585, P11=-0.14620822860604746, P12=0.08265691075590162, P13=-2.860768011968666, P14=2.645454389969472, P15=0.5340149919269597, P16=-0.7834976625999616, P17=3.0765641906867263, P18=1.6081453519016795, P19=4.191430336574227, P20=1.63128563657781, P21=4.865801117884803, P22=6.23657712045342, P23=6.481044473501929, P24=5.25633860938564, P25=6.441339237355789, P26=3.181523888781401, P27=4.172372132035343, P28=2.3327822987281643, P29=3.649493150561899, P30=0.6370042600579671, P31=3.5825881872019747, P32=0.3862405285864805, P33=1.507801934437574, P34=0.07358973599414607, P35=1.344533211777566, P36=0.43234424276224126, P37=1.5055623652723986, P38=0.10588792586832405, P39=1.605881015892992, P40=0.7314383533604859, P41=0.7004932087891063, P42=0.7166428701610478, P43=0.7224007341922793, P44=0.7454091757874259, P45=0.7545232353874363, P46=0.7334666536685092, P47=0.695742552423385, P48=0.6886798292189076, P49=0.7167353425708726, P50=0.7438987952160195, P51=0.745154016564934, P52=0.013634053137478692, P53=0.0013564315936185727, P54=0.00010906473463995421, P55=8.724178200468186e-05, P56=9.196346132783467e-05, P57=0.00040633189588014195, P58=0.0005433514025198645, P59=0.0012389635402504197, P60=0.009001670059126178, P61=0.025970002985484474, P62=0.03625503651970331, P63=0.0326666883206156, P64=0.015760929258506525, P65=0.008096400633020895, P66=0.004447475789456611, P67=0.0026513691158291217, P68=0.0020558616037202955, P69=0.004952883611971679, P70=0.013941604209460284, P71=0.013087371400179014, P72=0.017159276501804207, P73=0.029279118284863997, P74=0.02515557629325084, P75=0.02311575156791328, P76=0.023635670933309844, P77=0.02665237642986272, P78=0.01683338686455771, P79=0.016034523281982092, P80=0.00973322485987706, P81=0.0054509551828366976, P82=0.00807446576838034, P83=0.022059419949724327, P84=0.03282733345114488, P85=0.030816141292866722, P86=0.01808355198351056, P87=0.011974488462790872, P88=0.016979292630232876, P89=0.00976742052633236, P90=0.00814799983944177, P91=0.006945455240604677, P92=0.006336121594488846, P93=0.0052119838475534, P94=0.01771798788129361, P95=0.01675690369805168, P96=0.009322855310449041, P97=0.004728819545949189, P98=0.0014893491198839706, P99=0.0017894620092034866, P100=0.005527897886837255, P101=0.012889348745443632, P102=0.008985939195656227, P103=0.006819644046355515, P104=0.005231650508508341, P105=0.0018861889158139306, P106=0.0025503883205498542, P107=0.0032192867186926458, P108=0.001837774553332916, P109=0.0009816014549912852, P110=0.0008849126312542103, P111=0.0008142448642760765, P112=0.0007206552964860066, P113=0.0002626649979636711, P114=0.00018731564837858996, P115=0.000261399553474958, P116=0.00025041476597594084, P117=0.00021873149921533615, P118=0.00027698412934395913, P119=0.0001658043723286568, P120=0.00026295098198361597, P121=0.0002992936174206615, P122=0.00026853773209556625, P123=0.00029292621130657276, P124=0.00039128756832230527, P125=0.00022793549616971674, P126=0.0001404843569190385, P127=0.00011230570707860476, P128=0.0001075295260882897, P129=0.00014262813430319647, P130=0.00010966171412618963, P131=6.562940869290515e-05, P132=7.585859275050561e-05, P133=9.111078748567994e-05, P134=0.0001308455318874189, P135=0.00013737204933602937, P136=0.00010431845656545649, P137=5.089659536689584e-05, P138=7.873520280661423e-05, P139=0.00011289668651007125, P140=0.0001101852944501748, P141=0.00015613786613257876, P142=0.00011584551637802636, P143=9.943899158780917e-05, P144=0.00015470921744967705, P145=0.00012782635751316118, P146=9.378356986215746e-05, P147=5.920534384730131e-05, P148=3.862180937572003e-05, P149=4.1551797793740334e-05, P150=4.26601895723641e-05, P151=4.428378566267981e-05, P152=3.660533457580119e-05, P153=2.9799995918824813e-05, P154=3.963485025584438e-05, P155=7.008893278341847e-05, P156=9.631512241002751e-05, P157=9.214436923161458e-05, P158=5.838406545193258e-05, P159=5.167656385803454e-05, P160=4.0792972815983224e-05, P161=1.7794057844189168e-05, P162=1.8835180765544282e-05, P163=1.7850140577470383e-05, P164=1.750696042327462e-05, P165=1.6937485823793955e-05, P166=1.2127108416465085e-05, P167=7.351686445282835e-06, P168=7.0462992677290215e-06, P169=6.408555331075457e-06, P170=7.025717247593706e-06, P171=5.679437718060842e-06, P172=4.03293496020165e-06, P173=7.779847605539825e-06, P174=9.531948061822957e-06, P175=1.2445299614638594e-05, P176=1.1726429620995545e-05, P177=1.3176189270370543e-05, P178=4.245011482289025e-06, P179=8.997824896047925e-07),Row( P0=-516.809108743982, P1=32.4019896149286, P2=-15.364600693258577, P3=-9.61515738582023, P4=-14.181204993639888, P5=-10.821619505327483, P6=-8.054036211373049, P7=-3.4301107271645868, P8=-8.474228856120506, P9=-2.965928728301046, P10=-8.47896349763625, P11=-3.5055958123471207, P12=-1.1140773426855692, P13=-2.408331602480153, P14=4.919885155774833, P15=4.633292042309228, P16=6.410199532418457, P17=12.101793673453601, P18=3.5313609326831616, P19=8.287905947413847, P20=2.246136253100539, P21=4.38259563797919, P22=4.282373168001948, P23=0.8025725359898602, P24=-0.271101682515358, P25=0.3650887213360365, P26=-0.7022854252229479, P27=2.9803521859791644, P28=2.83895972583685, P29=2.6232582586262008, P30=0.9520543924128263, P31=3.2970306708115644, P32=0.697747701580868, P33=3.5268275825811144, P34=0.7879121533564912, P35=0.3776912638614916, P36=0.9271722515536857, P37=3.0497982718292285, P38=2.711396468195859, P39=1.7645075250132842, P40=0.784551319419943, P41=0.7565834000400128, P42=0.708699104707387, P43=0.6733414827542596, P44=0.6813331964242509, P45=0.6805840461885534, P46=0.6748782985569474, P47=0.6736541645996197, P48=0.6857163692832953, P49=0.7391370455758788, P50=0.7483834444494251, P51=0.7606899479610923, P52=0.0015441896268736414, P53=0.0002158191400876852, P54=3.555211926588e-05, P55=7.989613578457752e-05, P56=0.00010516722618963236, P57=0.00021517132406944357, P58=0.00026178323619293047, P59=0.00032029655923725944, P60=0.0007254185734110842, P61=0.004808300823552491, P62=0.026993522523404753, P63=0.06530451545226222, P64=0.04997781981826017, P65=0.10696365165878677, P66=0.18393159496360953, P67=0.33056166898772055, P68=0.506431043577659, P69=0.5009633364615365, P70=0.2512184005132039, P71=0.1036663637832814, P72=0.09789745922207109, P73=0.09430434432483342, P74=0.06677603532000952, P75=0.04476157770294361, P76=0.032162077605134225, P77=0.049021944620683634, P78=0.13916330946617125, P79=0.2749737203322914, P80=0.20635947410233754, P81=0.1268737576132376, P82=0.11230262357103721, P83=0.13833689869948063, P84=0.5348680550362708, P85=1.6618613343423612, P86=1.460559569146943, P87=0.861690100822007, P88=0.35227185612530976, P89=0.10046191305149871, P90=0.12477313434372928, P91=0.20598822860274482, P92=0.25323706142461644, P93=0.1419377760874383, P94=0.08267648682834342, P95=0.09377982191154653, P96=0.16717255103555612, P97=0.215983139980627, P98=0.2807473802863621, P99=0.26529020398055064, P100=0.24769566418632435, P101=0.1512023853433795, P102=0.03820662307224573, P103=0.017349642278051226, P104=0.03622408891410971, P105=0.0510897517231922, P106=0.04248644097797835, P107=0.035241751379926994, P108=0.02240464738467436, P109=0.01353364092932842, P110=0.01585827877654425, P111=0.020050312307806493, P112=0.022023169154787747, P113=0.016335396963639066, P114=0.020903357969146195, P115=0.008463688694163155, P116=0.005895293996893744, P117=0.005182916485390123, P118=0.017145985773073285, P119=0.023845912831709146, P120=0.003968425149476808, P121=0.011486484607497923, P122=0.02542791424320295, P123=0.011120809234630573, P124=0.007569893592858995, P125=0.014396654379610189, P126=0.01701006396669227, P127=0.011184288627324205, P128=0.021484615429297554, P129=0.02218504031439775, P130=0.014015248038570785, P131=0.01258979427708895, P132=0.005864608298125931, P133=0.0074085575392634564, P134=0.00296641862084354, P135=0.0030265730582488552, P136=0.004254757118054204, P137=0.0032053518431619794, P138=0.0015251890611632471, P139=0.00241463057496013, P140=0.0014485736371926728, P141=0.0021482602814795137, P142=0.0022194000159415465, P143=0.002000361252194596, P144=0.0026770359960156267, P145=0.0028085755363771343, P146=0.0017518925608873137, P147=0.0005287519361884976, P148=0.0002747007383846178, P149=0.00020619443746294202, P150=0.00023056769196771042, P151=0.00019659112285447489, P152=0.00015356545106522148, P153=0.0003694522305529579, P154=0.0003718044436118009, P155=0.00046523865425441197, P156=0.0005396887645289408, P157=0.0011415241183056646, P158=0.0017018666137325923, P159=0.0018106072207675007, P160=0.0021212892398039867, P161=0.0009729967821286677, P162=0.0006775033808240257, P163=0.0004702126446726066, P164=0.00020238707147411178, P165=0.00016760739493376214, P166=0.00014843942336678046, P167=0.0001542947281667308, P168=0.00019641966150547963, P169=0.0002829584790670582, P170=0.00029602411508359157, P171=0.00016657196576788368, P172=0.0002425118210203662, P173=0.0003895118984836248, P174=0.0003883015215155014, P175=0.0003499173821067148, P176=0.0003201086244698536, P177=0.00028356809956212747, P178=0.00012769522007618784, P179=2.0092655378498175e-05),Row( P0=-692.8934308055383, P1=59.30959421115109, P2=-13.767603606360282, P3=7.2452838410880265, P4=-5.461211913546758, P5=-8.708654354594826, P6=-1.8117448226434005, P7=2.6595724451855176, P8=0.9098185251526845, P9=-1.083769988358495, P10=-7.993738900490413, P11=0.84380187789127, P12=-0.525418380675147, P13=-2.2560175743769024, P14=4.116286475478138, P15=-3.0533000396316026, P16=1.8242031133326184, P17=3.7279130036330117, P18=-3.565199986223616, P19=3.3235481757509517, P20=-4.382141945099376, P21=-0.5541981766734367, P22=1.267591869443499, P23=-1.604776386690071, P24=2.020941818269568, P25=3.70013014129769, P26=3.4663947521872958, P27=3.79169256231125, P28=-0.06545721522145333, P29=0.3418440460635205, P30=-0.21519877660705436, P31=0.430761108697851, P32=4.334246220873134, P33=4.469939977974391, P34=4.426955701100918, P35=4.988076466372781, P36=0.7486405246620744, P37=3.8254509327845256, P38=3.193127926044793, P39=4.4592937227834, P40=0.7711163580963183, P41=0.6950445833173104, P42=0.6786905595744024, P43=0.7299087306753985, P44=0.8018626484168118, P45=0.7626996080111529, P46=0.681995276507616, P47=0.6570901771692684, P48=0.6833103302562424, P49=0.6956138876515512, P50=0.7416562563468349, P51=0.7813282060698822, P52=0.0001072721400909293, P53=2.5519111591839526e-05, P54=7.257390391958175e-06, P55=1.7082495733214292e-05, P56=3.002933410227335e-05, P57=0.0005580599539777819, P58=0.003198388277750546, P59=0.0046009311991798444, P60=0.006002215551665298, P61=0.01948694765936595, P62=0.015366104664168392, P63=0.002391699217713542, P64=0.0009610240060843386, P65=0.007451333795445918, P66=0.01750775530662097, P67=0.022899788592269623, P68=0.010439771439203488, P69=0.0031640818563600445, P70=0.006078906654925621, P71=0.012793290547706433, P72=0.0218444720089175, P73=0.012718739634998377, P74=0.004456100106967878, P75=0.003845561528555711, P76=0.0029655225592314914, P77=0.0022565125628771633, P78=0.002294915361661261, P79=0.0032298426156851397, P80=0.007225699159947053, P81=0.011423432975446304, P82=0.027794524945310446, P83=0.027888549082255982, P84=0.026655295581940776, P85=0.018770168113427705, P86=0.005430084483411723, P87=0.010630933932714429, P88=0.012951294207702014, P89=0.011875777935174947, P90=0.02170755722004297, P91=0.022218248424751178, P92=0.04492639584584498, P93=0.01836545622352956, P94=0.002041138477792282, P95=0.0031229469380949274, P96=0.028145489058784987, P97=0.05027933941013321, P98=0.024344901130249022, P99=0.0029727016119766633, P100=0.000432833058448568, P101=0.0017987158009499103, P102=0.004043542525526508, P103=0.002658450121794121, P104=0.0008127846560927762, P105=0.0006503112469449222, P106=0.002328906180167291, P107=0.005844888413535468, P108=0.0020570964105529944, P109=0.0008154151936587935, P110=0.019627972303901606, P111=0.02561707536068386, P112=0.008662393014951595, P113=0.0024264916527424805, P114=0.00270875159930122, P115=0.0022529860791988426, P116=0.0019192026301231577, P117=0.0012131458174436777, P118=0.000888656126463177, P119=0.0011332876480827738, P120=0.0009469723958166187, P121=0.0007150762150028448, P122=0.0012848662519227194, P123=0.000751146500062701, P124=0.00030287416964174164, P125=0.001131499982026669, P126=0.000611688826320817, P127=0.0006165714085634583, P128=0.00037808437299965216, P129=0.0002855719959063333, P130=0.0003540173943804119, P131=0.00031140561919799144, P132=0.0004074495710082658, P133=0.00037798471119637696, P134=0.00025433840922533895, P135=9.255107289859526e-05, P136=0.0001384332042554639, P137=6.894503773908913e-05, P138=6.474891828236296e-05, P139=6.326320822224764e-05, P140=6.823138056371573e-05, P141=8.653079952803335e-05, P142=0.00013717599362233931, P143=0.00015762836015156747, P144=0.00019071585008327915, P145=0.0002974090215898629, P146=0.00040218441860172916, P147=0.00022337233455875272, P148=6.6687710948706e-05, P149=6.495995464034463e-06, P150=1.3725385223144978e-06, P151=2.0644218809530313e-06, P152=3.0243905641496583e-06, P153=5.180998531552191e-06, P154=7.67423501183606e-06, P155=7.708816768170883e-06, P156=8.06179017669304e-06, P157=5.883554240308365e-06, P158=8.731831879840298e-06, P159=1.5129748840116191e-05, P160=1.085542408440344e-05, P161=5.294258000066454e-06, P162=7.2025496460438585e-06, P163=7.717933521030301e-06, P164=3.0756790553255258e-06, P165=2.700388973363594e-06, P166=2.1439250009376587e-06, P167=1.212053748346339e-06, P168=1.8169864368655638e-06, P169=3.879712543652089e-06, P170=4.655588264880365e-06, P171=2.7403137949764017e-06, P172=1.2356101550611875e-06, P173=1.326971115888335e-06, P174=1.6112440137686917e-06, P175=2.0166833039183944e-06, P176=2.752071121419515e-06, P177=2.317479158756079e-06, P178=1.1544934989433383e-06, P179=1.3587706153861947e-07),Row( P0=-569.5996301760127, P1=34.026582828878205, P2=-20.801016500335216, P3=-8.760926332413023, P4=-14.98529619950579, P5=-8.951433008467028, P6=-13.295034928076863, P7=1.2292993261427219, P8=-7.601989065478035, P9=-5.220883834996325, P10=-7.62168427949867, P11=-3.9667246334740844, P12=-0.5781701898840914, P13=-3.2881305565563568, P14=4.15391522352968, P15=5.179076504354197, P16=7.730693895184852, P17=12.205884254595013, P18=4.18491562350594, P19=7.453425882468936, P20=-3.413295225708801, P21=0.3675598348373353, P22=2.3678035200526963, P23=-0.20731179680867515, P24=2.2120270774329445, P25=2.4552317175045943, P26=-2.1460667202379375, P27=-0.7472231820370578, P28=-0.4248739566914752, P29=2.9846280897883792, P30=1.57672374591394, P31=3.7514811706131264, P32=0.5017041692575255, P33=2.6649438203480313, P34=3.840341563141221, P35=4.276462159682743, P36=1.974521055426032, P37=0.30806847456522224, P38=-0.03840636582765636, P39=1.6230850069190859, P40=0.7960904128455984, P41=0.695190106302725, P42=0.6591901790455417, P43=0.6648763987092619, P44=0.6440111636364801, P45=0.6691733705844377, P46=0.6896376349150949, P47=0.642431494910388, P48=0.6208841513747978, P49=0.6254533601017158, P50=0.6869507575315293, P51=0.7600735355180939, P52=9.545150856971414e-05, P53=2.9745393105307526e-05, P54=1.336001872052215e-05, P55=2.1927525959254643e-05, P56=4.718550169841745e-05, P57=0.00020965587801963074, P58=0.0012009900249252393, P59=0.0027167277278738375, P60=0.001790100680162053, P61=0.0035111139376611078, P62=0.008966537939893805, P63=0.014312605469058763, P64=0.014268718551079248, P65=0.04850667783544649, P66=0.19601371549059102, P67=0.33338213235759073, P68=0.15675487209682604, P69=0.0623361791872479, P70=0.04665729914836528, P71=0.020087157868128067, P72=0.010831233388223619, P73=0.010664140252382508, P74=0.008202362742642062, P75=0.007432777266011859, P76=0.006323744530361904, P77=0.01095257761581904, P78=0.016972812583016986, P79=0.02670060200871705, P80=0.05171548217965304, P81=0.10734241367455281, P82=0.19802008647534816, P83=0.3604748783562055, P84=0.44385007416811256, P85=0.5866894845615944, P86=0.34168353769430787, P87=0.2927609473197692, P88=0.3755661992746952, P89=0.26931893162107223, P90=0.21333917706460867, P91=0.1754245408172788, P92=0.3505982807409776, P93=0.31030200616251924, P94=0.4740000355751802, P95=0.3729895734034651, P96=0.18908229300757856, P97=0.27591263961706336, P98=0.21770770527976385, P99=0.1276236410591402, P100=0.09793388452813495, P101=0.07513455201651953, P102=0.08586862291656722, P103=0.03874462115450594, P104=0.021417237682653516, P105=0.056519292820398394, P106=0.056670398273643494, P107=0.031143487870980688, P108=0.015242706902849948, P109=0.022258480620814265, P110=0.0569003617214683, P111=0.07619838272392972, P112=0.04049584288484953, P113=0.019084852946343215, P114=0.019481290454653806, P115=0.02653080139102837, P116=0.01054009897693835, P117=0.012512257593879896, P118=0.007214377874790825, P119=0.003464956128226177, P120=0.017068651347780472, P121=0.012709652054831634, P122=0.016064043114872195, P123=0.008883580132957213, P124=0.005107401982284553, P125=0.004690174453130492, P126=0.006376778635188576, P127=0.004319348465108468, P128=0.0011417705911818529, P129=0.001139575817871036, P130=0.0024117233423495343, P131=0.0020672621652438935, P132=0.0024353023637622376, P133=0.004723485023993252, P134=0.002811553505516966, P135=0.002262179601635113, P136=0.0022938854136159777, P137=0.0008128848223103659, P138=0.0006053710661046927, P139=0.0004398735807622039, P140=0.00034770210787751484, P141=0.0004270580193875253, P142=0.0005113566686505515, P143=0.0008024910685062161, P144=0.0009043784206914691, P145=0.0016618821456392042, P146=0.0017356316008911477, P147=0.0013351661226302998, P148=0.0006075059241843312, P149=0.00025130097207208497, P150=0.00018054115241309844, P151=0.0002342819661744644, P152=0.00011286047564407702, P153=0.0001494562236266224, P154=0.0003173741182180272, P155=0.00045267129132799334, P156=0.0009364565063001163, P157=0.0014403478370033332, P158=0.0012436314368874464, P159=0.001298404197313952, P160=0.0010354271385443284, P161=0.0006276042542121174, P162=0.00031578479697835906, P163=0.00017205802709544438, P164=0.0001291947685625992, P165=0.00010327690003547171, P166=0.0001189402052880438, P167=0.00010716383098530034, P168=8.408955381464921e-05, P169=7.717718216441276e-05, P170=6.960800832999216e-05, P171=5.457433971674125e-05, P172=6.586997553047693e-05, P173=0.00012708368064952822, P174=0.00010303831704504876, P175=5.8678864281249063e-05, P176=5.361871481310924e-05, P177=5.9186933773354224e-05, P178=3.469172312031067e-05, P179=3.686038138229732e-06),Row( P0=-681.1479146236632, P1=40.63906373328796, P2=-31.54906479815036, P3=4.722114862174375, P4=-17.970327202793694, P5=-17.419064423237604, P6=-1.7535326384562315, P7=-3.923187942319253, P8=-4.336406123939813, P9=-13.621953138224793, P10=-4.175269265295806, P11=7.340660688867425, P12=-1.2743878177769807, P13=10.129136649189892, P14=15.537781710362346, P15=9.306711329723788, P16=5.695410244521139, P17=9.548012226771373, P18=6.46625057164056, P19=10.165891393637336, P20=6.064179974463634, P21=7.8028285918498295, P22=-0.13734811092245933, P23=-1.8507996544250542, P24=2.544963060144519, P25=-0.10488596976482066, P26=0.17258386863629366, P27=2.4281595107513856, P28=-3.174673217943157, P29=1.0764126507058118, P30=-1.8189839009550057, P31=2.805212371707067, P32=-0.5794523813313377, P33=1.4740914780732997, P34=-1.1575572422774107, P35=0.5945577814291078, P36=-2.78254944740744, P37=1.2143770705894328, P38=-2.7567181098225064, P39=1.1962740147587319, P40=0.7822644743231754, P41=0.7044220982747181, P42=0.639522132243338, P43=0.5951465105308572, P44=0.6078250160840107, P45=0.6051384813001559, P46=0.5587004326719567, P47=0.572951749822272, P48=0.553797678638436, P49=0.598943936143235, P50=0.6868525098802557, P51=0.7423407973856572, P52=1.4080866457249332e-05, P53=7.399750807274531e-06, P54=6.196872918841749e-06, P55=1.8827380796533705e-06, P56=1.387313415135338e-06, P57=1.038844866208521e-06, P58=4.253268455523543e-07, P59=2.970241304152162e-07, P60=2.1278011260799115e-06, P61=1.037090962442352e-05, P62=0.00026584804302343226, P63=0.0009802986388793864, P64=0.0010504560284428277, P65=0.0033026752193471634, P66=0.009782198631144913, P67=0.018533026889108437, P68=0.01289925122994964, P69=0.019622917880285452, P70=0.023621375118515083, P71=0.014519562781293623, P72=0.015620963002283465, P73=0.008872557685289814, P74=0.0028590694676676257, P75=0.002651550695436762, P76=0.0008635577718460238, P77=0.00035440118208432777, P78=0.0004306178364829314, P79=0.0005986585538971757, P80=0.000805843316215833, P81=0.0009045490431278397, P82=0.0019285071918858042, P83=0.003358937557681688, P84=0.003841293578574401, P85=0.004321451920045883, P86=0.004838289527156697, P87=0.004206205387201693, P88=0.006805858388834482, P89=0.006080993623640577, P90=0.004391677788950987, P91=0.009064329388578199, P92=0.012688768573010432, P93=0.003504706525133082, P94=0.001712505360381017, P95=0.004114442627147708, P96=0.0027635316370477107, P97=0.0008779464114638314, P98=0.0040979637784985794, P99=0.0062033748735842945, P100=0.002184138879845612, P101=0.004086144607993223, P102=0.0018257470325217828, P103=0.00022883402382819248, P104=0.00124824722794575, P105=0.0008625259934622652, P106=0.0001729832516351393, P107=0.0002732261492480188, P108=0.0008844377081299629, P109=0.00038752003176265314, P110=9.182297467945674e-05, P111=0.0009303679954739795, P112=0.0010618504115263692, P113=0.0004449983334259421, P114=0.00045071684060281217, P115=0.0008359529304086634, P116=0.0008367213844909018, P117=0.0008961510878323541, P118=0.0006161558575869875, P119=0.0005016360393239153, P120=0.0007424962291006085, P121=0.0016607997317914706, P122=0.0009413703970633016, P123=0.0005238012298267516, P124=0.0004945755057091419, P125=0.0012660808907109548, P126=0.0005411109869576668, P127=0.0001719185380441496, P128=0.00037995799747655046, P129=0.0008381892305884318, P130=0.000180381299267675, P131=4.270097743973407e-05, P132=4.544289289929532e-05, P133=7.257708240274239e-05, P134=0.0001002520410412768, P135=6.47772008474596e-05, P136=7.650680825715684e-05, P137=5.7668922267367095e-05, P138=8.174637154757123e-05, P139=7.419571832358592e-05, P140=2.2266480723878117e-05, P141=3.672376284262638e-05, P142=2.193370656153292e-05, P143=1.9854035667421202e-05, P144=3.3147360901722e-05, P145=2.080457281414988e-05, P146=3.8492848275327195e-05, P147=5.2195042969421225e-05, P148=3.5969382711179534e-05, P149=3.098703564904622e-05, P150=2.3284563393692206e-05, P151=1.4820483626420331e-05, P152=1.385874415074124e-05, P153=5.282328949391452e-06, P154=7.217796223916289e-06, P155=1.2679328921420467e-05, P156=3.580033169708179e-05, P157=4.218432507495297e-05, P158=3.511075921756539e-05, P159=2.9296561398429177e-05, P160=4.955061843774103e-05, P161=2.4566189985342984e-05, P162=4.288017490919114e-05, P163=2.0816144358351887e-05, P164=9.090640459694211e-06, P165=6.784940090058441e-06, P166=8.931762809359735e-06, P167=1.115119200774169e-05, P168=6.447735388401282e-06, P169=8.852132206010756e-06, P170=1.1957184161164216e-05, P171=1.6573004346468103e-05, P172=1.7738970528638387e-05, P173=2.0061992874246673e-05, P174=3.4622970562195576e-05, P175=6.035682005468046e-05, P176=8.698508722930132e-05, P177=0.00013403638387822028, P178=3.810990931780837e-05, P179=3.0050053604369703e-06),Row( P0=-673.777043381718, P1=46.49290810041964, P2=-12.61233572676063, P3=-5.974627439347443, P4=-7.396656202396246, P5=-0.9615170682059737, P6=5.853392648266628, P7=5.731341426965173, P8=-2.486922597275131, P9=-1.8969897257693347, P10=-3.9737044122958727, P11=2.388192793798503, P12=5.433167012447002, P13=15.955928514459703, P14=16.70325166285633, P15=19.982698832336478, P16=13.484752471351232, P17=5.87336850191202, P18=3.0284782724962045, P19=2.6019198815035023, P20=6.343132150612938, P21=6.864733903951208, P22=1.4695900531450554, P23=-2.49935329008412, P24=-0.48654000132918745, P25=1.0779917352259172, P26=5.235636387777057, P27=7.442385986613502, P28=2.6956621594502304, P29=1.7636507971709265, P30=2.1259996390842115, P31=2.331380594034195, P32=-0.3890368436180758, P33=-1.2662550786039963, P34=-3.7719273391085095, P35=1.949866495413974, P36=-0.407346002501592, P37=-1.1844914493912364, P38=-3.950297081599183, P39=-0.13054146868268018, P40=0.6116355466593956, P41=0.6442476568752034, P42=0.7507567871728086, P43=0.7549414286244563, P44=0.7070955647143675, P45=0.6397085456202075, P46=0.5944392415765941, P47=0.5707881158188048, P48=0.5263416310853978, P49=0.534224896757113, P50=0.5797854119198167, P51=0.5791075004885723, P52=0.00028797440817601515, P53=0.0007168519170991432, P54=0.00020402079580898825, P55=4.2205107618787e-06, P56=2.733201288194459e-06, P57=7.463291668467148e-06, P58=4.402557280398022e-06, P59=2.2335938094836045e-06, P60=4.182935367617743e-06, P61=9.879520860059283e-06, P62=1.9366669550139684e-05, P63=3.680242059157036e-05, P64=4.46877163411435e-05, P65=0.0003060609065257462, P66=0.001493421338725362, P67=0.0031718512714703755, P68=0.004063961380802187, P69=0.012838477914962614, P70=0.017959030543471238, P71=0.009885345826961524, P72=0.0031808813077686205, P73=0.00058915005683114, P74=7.301995116671555e-05, P75=5.732569336158221e-05, P76=8.673844828121129e-05, P77=0.00014199556427107053, P78=0.00028043266651629447, P79=0.0005508919582457735, P80=0.0007129230449515748, P81=0.0015227178361538593, P82=0.0014543253317490696, P83=0.001804865768945046, P84=0.006326136853009129, P85=0.02835936653999218, P86=0.0255700264192334, P87=0.039019995653102324, P88=0.05951941365717002, P89=0.03454018435719627, P90=0.033353350549418494, P91=0.03619639675064646, P92=0.0196613606925217, P93=0.022998522531712605, P94=0.022577193871207532, P95=0.008657910381622776, P96=0.011850354480847041, P97=0.005876368175124508, P98=0.003245694708657084, P99=0.0035858526168503885, P100=0.029140504758388447, P101=0.06444660031776701, P102=0.017787096560642136, P103=0.007051855562374041, P104=0.0009405437788772964, P105=0.001130057481795777, P106=0.0009065552157811184, P107=0.0006164436787133731, P108=0.001163877876561395, P109=0.0020038666143406747, P110=0.0015902534700180573, P111=0.0004992367689378679, P112=0.0004974158817334078, P113=0.0007631080136353497, P114=0.0005150361912473322, P115=0.0008945897377940969, P116=0.0011791074681179751, P117=0.0011507109218498257, P118=0.0005236634088661021, P119=0.00020437238463346386, P120=8.293812939080379e-05, P121=0.00023950410621835887, P122=0.0003255932707687654, P123=0.00015087405617964495, P124=8.953709904440909e-05, P125=0.00011422484844751305, P126=0.0004866712422546076, P127=0.00021250583963615996, P128=2.6137798213565582e-05, P129=8.332287466039154e-05, P130=0.00038958976792256714, P131=0.0003456733971554354, P132=0.00010035254743186857, P133=5.072325648166465e-05, P134=0.00014984299779949233, P135=4.167172896191949e-05, P136=0.0001128045253944352, P137=0.0002368532927545908, P138=7.44443115392799e-05, P139=1.932496949746401e-05, P140=3.4560483178152736e-05, P141=8.489378788556832e-06, P142=7.670926376640309e-06, P143=9.443765190396078e-06, P144=7.642113699605016e-06, P145=1.0346185862621233e-05, P146=1.87901572232917e-05, P147=1.906099261850098e-05, P148=1.3862595685492758e-05, P149=9.597692142918415e-06, P150=7.928980470378002e-06, P151=9.574589778047848e-06, P152=9.738972688926004e-06, P153=1.0613353767396479e-05, P154=1.700623407056028e-05, P155=2.1996403836162787e-05, P156=3.213494648954375e-05, P157=4.46378027243569e-05, P158=4.3156041585313506e-05, P159=3.737486286107389e-05, P160=5.217462302137245e-05, P161=5.033596207431243e-05, P162=3.18011990123573e-05, P163=2.92279107301644e-05, P164=4.3838586744424005e-05, P165=4.422418692314934e-05, P166=5.642346794606753e-05, P167=0.00010698671936757652, P168=0.00011028087909865674, P169=0.0001618054929149568, P170=0.00014910661098805342, P171=0.00012052881062255875, P172=3.6270843790981703e-05, P173=2.0637379745152945e-05, P174=4.055615991584088e-05, P175=8.770924291647125e-05, P176=0.00014066030562407986, P177=0.00015902619696767274, P178=8.245717094231693e-05, P179=7.7011546871606e-06),Row( P0=-474.6508909900806, P1=-8.726126500336907, P2=-62.2256815894383, P3=-1.8605361945887975, P4=-18.802960694178527, P5=-12.494441411992566, P6=14.530603258611752, P7=23.09501513154102, P8=6.1096406135486045, P9=2.5113691275140124, P10=-14.1483533885246, P11=7.548027483077409, P12=0.11726471919793774, P13=15.441845321089572, P14=6.159852538959688, P15=-7.421804688346812, P16=1.6648948498028215, P17=7.057848901965004, P18=11.97339275741049, P19=4.0789279472652, P20=9.746182302458301, P21=11.765953837449663, P22=0.29280586863895325, P23=-2.0188932653928284, P24=7.487885486400239, P25=5.772424358563284, P26=0.5343532403374771, P27=3.193904169013849, P28=3.076780441091691, P29=5.267133504926651, P30=-0.01921982535009682, P31=3.487000631398836, P32=3.2757098974889285, P33=3.1610345698805133, P34=3.4577189408335363, P35=5.3653629692877445, P36=-0.6780080434630779, P37=1.922509294388607, P38=-0.20744413007556511, P39=2.7607023312612737, P40=0.6027860319297512, P41=0.4947054975126083, P42=0.4466253812480453, P43=0.4980854245350966, P44=0.5598572133340723, P45=0.5705373397794947, P46=0.5160947121281053, P47=0.5283261383634414, P48=0.45946314999232024, P49=0.45902233819073257, P50=0.5450815078085527, P51=0.6183886496468951, P52=0.012147834449739608, P53=0.0008641920219888672, P54=2.4567644060200074e-05, P55=2.1215283543262924e-06, P56=3.2088910687716907e-06, P57=3.241097870349561e-06, P58=1.4903273849828648e-06, P59=2.0452863188995247e-06, P60=8.091781050809335e-06, P61=2.1018247891848634e-05, P62=4.219752882196149e-05, P63=5.686733123501618e-05, P64=3.134460147981142e-05, P65=3.4965027753942544e-05, P66=8.275610009776605e-05, P67=0.0001525547239361016, P68=7.114706731926083e-05, P69=3.171796729542568e-05, P70=3.8272677017096114e-05, P71=0.00011461492513950322, P72=0.0007805818876766052, P73=0.0011459764868741698, P74=0.000885902514314613, P75=0.0024960131545555845, P76=0.0044354962694975525, P77=0.006171852767303981, P78=0.007368279754950935, P79=0.007538975647717647, P80=0.008428563531265846, P81=0.015556562824161459, P82=0.021467927782255765, P83=0.017126138857345933, P84=0.027958390581668973, P85=0.057783937692819115, P86=0.04358733364961861, P87=0.05486703743825612, P88=0.1909164906468388, P89=0.5892626921986971, P90=0.6427994310528681, P91=0.906785719717236, P92=1.7310849243890039, P93=2.4513382238359216, P94=1.5226640190201166, P95=1.744629445257173, P96=3.198084893207565, P97=1.5379167070204394, P98=0.11369680028381253, P99=0.03804151484536412, P100=0.023585202532359798, P101=0.029208302411984015, P102=0.023456494505408692, P103=0.01701931399004707, P104=0.013416955158344507, P105=0.012898766807823918, P106=0.036200256611248566, P107=0.08142922140300887, P108=0.04832414087582677, P109=0.046190371404748, P110=0.08755720413788003, P111=0.14896536712518124, P112=0.28397429096452875, P113=1.2891488807484783, P114=1.2856314461351335, P115=0.33792899868296544, P116=0.5940258387556785, P117=0.5243498316099311, P118=0.5376450629947364, P119=0.1250735366029685, P120=0.027050162350455723, P121=0.0918344643384564, P122=0.15105190512370154, P123=0.17762445117845369, P124=0.3694240049291433, P125=0.5353764880545365, P126=0.3983531807418936, P127=0.2043921914250312, P128=0.13805126428457815, P129=0.1083171915396409, P130=0.20851737703409812, P131=0.0551883986420123, P132=0.030923521597539225, P133=0.04073662478137912, P134=0.046082542724376875, P135=0.06704364101105842, P136=0.04284217543227993, P137=0.043936192144866215, P138=0.02162608505365283, P139=0.04075585247751572, P140=0.02657236157024142, P141=0.010041410601015134, P142=0.008519885855810374, P143=0.0070874926867686445, P144=0.0056902479904294015, P145=0.011838052717738745, P146=0.016517850537416005, P147=0.006248241539295029, P148=0.0032491958110752632, P149=0.004953549423687676, P150=0.006150639907492129, P151=0.006293098071034018, P152=0.0008322935150084476, P153=0.00023533687366862383, P154=0.00018252806712492408, P155=0.00012437664241974714, P156=0.0001842716921759041, P157=0.0001658798566981482, P158=8.929442614635063e-05, P159=0.00029179539731737585, P160=0.0003590859535384397, P161=0.00039836971352888255, P162=0.0006591357072711888, P163=0.0006574084096330475, P164=0.0007431276211927768, P165=0.0006798772170555498, P166=0.000615428263317488, P167=0.0005644965488107778, P168=0.0003644413054813413, P169=0.0002579475489317597, P170=0.00023685222324759444, P171=0.0001237654069613982, P172=5.1856814244158764e-05, P173=5.155236460691356e-05, P174=6.0470354361738346e-05, P175=6.493851094777042e-05, P176=9.439523138465123e-05, P177=0.00011842573699986971, P178=7.136009724295469e-05, P179=5.1031349786010494e-06),Row( P0=-747.2668547756538, P1=37.29023176370067, P2=-16.790422086773347, P3=-14.832365287220657, P4=-15.242246123702799, P5=-17.251937389793504, P6=-8.55579788456654, P7=0.0075441099960704356, P8=-10.682819214267818, P9=-11.063237366216507, P10=-5.038958610192125, P11=-6.1461765821807495, P12=-10.571508542611031, P13=-1.2733033378178933, P14=-4.568005249504673, P15=2.5231823110914133, P16=5.729837579273547, P17=9.480189842153282, P18=14.22821064120728, P19=13.311992983636532, P20=10.424803714478564, P21=7.947286094811007, P22=-1.099601359901574, P23=0.9779288042734477, P24=-0.9695354982580748, P25=1.3218141658672837, P26=-1.2525898760504561, P27=1.137001133246176, P28=-3.2371507737075893, P29=-2.803960787127451, P30=-0.8678683208809449, P31=-0.09693767091280346, P32=-0.21776203761787957, P33=1.7026452806968015, P34=-4.399527730573039, P35=-0.939614987164743, P36=-1.7019257907324286, P37=1.2212911641318163, P38=-0.9780558881019574, P39=-1.8298167574090174, P40=0.5585489872100282, P41=0.5230712392464867, P42=0.4697818596866934, P43=0.49596234790664734, P44=0.5731329329695942, P45=0.5113929261941849, P46=0.46507269945043406, P47=0.4900970056515467, P48=0.5132874387363666, P49=0.621424163724379, P50=0.6127579803507687, P51=0.5841193711696534, P52=6.912380345449922e-07, P53=9.437228902171466e-07, P54=3.7665329364066937e-07, P55=2.3931360590664103e-07, P56=6.223826762420843e-07, P57=1.1701223932226518e-06, P58=4.344759099823436e-07, P59=7.774172282968324e-08, P60=6.190021025330872e-07, P61=5.530939378578234e-06, P62=9.748536054396661e-05, P63=0.00030069986509166473, P64=0.00046082640367010213, P65=0.0023384930887776142, P66=0.0026449660858660078, P67=0.0024615876706836327, P68=0.001207213749056838, P69=0.0002468238275881401, P70=6.040737838465045e-05, P71=1.2610841770365139e-05, P72=5.701086064681233e-06, P73=8.587391565805782e-06, P74=6.754539319870198e-05, P75=0.00013672644196752756, P76=0.00021461597237587042, P77=0.0011247302169858646, P78=0.0018416547647251366, P79=0.0038351553271002326, P80=0.003953563705583298, P81=0.0024442190400269866, P82=0.0041106675293848816, P83=0.002704579900618119, P84=0.0010488206417120657, P85=0.0011804323586803885, P86=0.0006284632013032702, P87=0.0002646618800635015, P88=0.00045480590725820603, P89=0.0023573579746954103, P90=0.0019504586266379637, P91=0.003155379256893696, P92=0.004447774832904802, P93=0.0006409592702507059, P94=0.00022373463324621326, P95=0.00030715867224790294, P96=0.00011732106308092321, P97=7.25228816213416e-05, P98=0.0001848308995421508, P99=9.499415565774547e-05, P100=0.0004671597695250016, P101=0.0009866375901856012, P102=0.00019833357454996595, P103=3.570400761504951e-05, P104=4.3694496217896273e-05, P105=7.253185239088891e-05, P106=3.164086143052952e-05, P107=7.193587270116958e-05, P108=8.044869179919935e-05, P109=2.3050874508972698e-05, P110=2.5835381299441334e-05, P111=1.1968637232532213e-05, P112=4.777037777259828e-05, P113=8.171945220570022e-05, P114=1.875441882844943e-05, P115=6.862354327780214e-06, P116=3.0105902676050275e-05, P117=0.0001129426160788419, P118=0.00011940191153795332, P119=2.263918745209616e-05, P120=4.639248183862938e-06, P121=3.8929612450412596e-05, P122=8.483795458405126e-05, P123=2.0436643130609995e-05, P124=9.518348492195646e-06, P125=3.432236199833797e-05, P126=6.725014965599609e-05, P127=8.603372371911677e-06, P128=2.096066186071008e-05, P129=7.154854735192867e-05, P130=2.1099451583879617e-05, P131=1.6620293308030624e-05, P132=2.9376589320725945e-05, P133=3.425216003626718e-06, P134=2.123133274887352e-05, P135=1.631942605373273e-05, P136=5.11204239818823e-06, P137=1.2342681302238536e-05, P138=3.690690330842115e-06, P139=7.390604263823817e-06, P140=1.3259547251999113e-06, P141=1.1048122245657488e-06, P142=4.874102878627959e-07, P143=5.369407751947672e-07, P144=7.740059732353677e-07, P145=2.0641893503300284e-06, P146=3.4439594203834863e-06, P147=3.5207305565877603e-06, P148=2.0215013629742858e-06, P149=3.0691354768720614e-06, P150=3.97125428077506e-06, P151=4.8278279021406265e-06, P152=5.904201237562571e-06, P153=6.253333117021128e-06, P154=7.036234217939958e-06, P155=9.716436359540086e-06, P156=6.793965434898331e-06, P157=3.4724053058434357e-06, P158=3.7903517006776533e-06, P159=1.1870571439876073e-05, P160=5.2535422891043965e-05, P161=6.682982506128037e-05, P162=5.268959077014564e-05, P163=9.593194571888982e-05, P164=0.0001284701873501482, P165=7.455518922464308e-05, P166=5.8484721657785956e-05, P167=4.5857315711637346e-05, P168=3.0072255018066654e-05, P169=4.6757530591426684e-05, P170=8.935800977240056e-05, P171=0.00011838331221022002, P172=9.691017320090452e-05, P173=3.96316493890294e-05, P174=5.264523561512765e-05, P175=9.279820307403814e-05, P176=0.00011364436292733059, P177=0.00011476311646103804, P178=4.205515025412406e-05, P179=3.745596522934721e-06),Row( P0=-725.718698726155, P1=47.3979238931378, P2=-21.470776876194655, P3=0.31808756457124604, P4=-18.58167276905142, P5=-16.427597887395752, P6=-0.5386802430819407, P7=0.09634934376278027, P8=-9.690003981460029, P9=-12.952187686777815, P10=-4.751857814075242, P11=-4.404415994091862, P12=-6.311563403389189, P13=-0.9881139929008314, P14=0.9269496351991269, P15=8.07855460919431, P16=6.262207741677222, P17=10.112715745909616, P18=11.127231871601026, P19=16.25341304902311, P20=11.528423371019244, P21=12.48023283026078, P22=9.041998415424931, P23=1.16262961966396, P24=0.027074963341286472, P25=-3.1154653318026946, P26=-1.7580000514716467, P27=3.1862184151903237, P28=0.8920411732952098, P29=4.7340107950285155, P30=-1.8722000299464594, P31=0.8826141097453052, P32=-1.1882699361512217, P33=0.9831348151084018, P34=-4.945516141420519, P35=-1.1435560619513612, P36=-4.366529673291966, P37=1.8126421626415838, P38=-2.390121143148983, P39=1.5544627227056196, P40=0.6296706628394958, P41=0.6495403991557931, P42=0.6470446977908236, P43=0.58243536469616, P44=0.5576620229768372, P45=0.5360695396443377, P46=0.5697866870916326, P47=0.6834001720273712, P48=0.7238500042316622, P49=0.665153275996002, P50=0.635514485839538, P51=0.6435174041306715, P52=9.842643219162561e-05, P53=1.0398171353379519e-05, P54=9.73954194527115e-07, P55=4.591991354567376e-07, P56=8.48064569490074e-07, P57=4.993890366439173e-07, P58=8.467734023140613e-08, P59=2.9653647843675244e-07, P60=1.839618188248128e-06, P61=7.4769720765026205e-06, P62=0.0001062001060135669, P63=0.0008005850496730212, P64=0.002601626843594237, P65=0.003233455133569875, P66=0.001373743941788189, P67=0.0014510667121739522, P68=0.004981602229207445, P69=0.012862552749413841, P70=0.006603189402747036, P71=0.0005083974267364555, P72=9.568214301919086e-06, P73=5.1962048063858195e-06, P74=8.171321651731456e-06, P75=2.1604678412424543e-05, P76=0.00012558705436427597, P77=0.00023095299417505368, P78=0.0003216943981884936, P79=0.0003886064737201801, P80=0.0001644846899211143, P81=3.391905130838023e-05, P82=4.5112537371435453e-05, P83=4.515147561417402e-05, P84=0.00021229837914938223, P85=0.0027986171555806696, P86=0.007541260810865111, P87=0.007724620735182469, P88=0.0028681972815798374, P89=0.00028906785146039444, P90=7.885632436008361e-05, P91=0.00012743153419847005, P92=0.00015028063595128515, P93=2.4415523532584434e-05, P94=1.7122804572533603e-05, P95=2.8656392020455467e-05, P96=2.2256803662565416e-05, P97=0.0001664753255857603, P98=0.000502889497955466, P99=0.00032646125064459883, P100=8.505896730165797e-05, P101=5.615707845758332e-05, P102=2.051712691819665e-05, P103=1.1107725897507873e-05, P104=1.5008042641989223e-05, P105=9.60205036354029e-06, P106=2.9722027907182835e-05, P107=0.0001549823605107622, P108=5.655166575404503e-05, P109=1.1688429307403683e-05, P110=9.237657313049283e-06, P111=4.219410567951867e-06, P112=1.8497078410731053e-05, P113=5.201135584425429e-05, P114=1.3290724071606812e-05, P115=1.1133781629941692e-05, P116=6.651002291347718e-06, P117=1.345392712830814e-05, P118=2.1245119883049716e-05, P119=1.6138333262582468e-05, P120=1.206155796308255e-05, P121=1.7553369226432763e-05, P122=4.403312663672751e-05, P123=2.017869558211602e-05, P124=2.2983035600342895e-05, P125=2.1977668448963987e-05, P126=1.599320035514238e-05, P127=6.846913089609741e-06, P128=9.120480026674498e-06, P129=1.653727195432892e-05, P130=1.2051426480094338e-05, P131=1.2338459116321095e-05, P132=1.2544546732104492e-05, P133=2.1003613886778714e-05, P134=1.7145343880322244e-05, P135=2.5278380693915967e-05, P136=1.8212455799617963e-05, P137=2.121672822854792e-05, P138=1.1931510256348668e-05, P139=2.521494311689442e-05, P140=1.6849044660341025e-05, P141=1.6117497301886372e-05, P142=1.0922124270395018e-05, P143=5.63721422748676e-06, P144=5.431580230482675e-06, P145=4.677459502164185e-06, P146=6.462381271308479e-06, P147=1.0616748304638741e-05, P148=1.575596391469933e-05, P149=1.707060815956299e-05, P150=3.4115792275225116e-05, P151=9.601818411146773e-06, P152=3.849098856376328e-06, P153=5.10847654226792e-06, P154=8.819450175769107e-06, P155=1.0796573354760919e-05, P156=1.3014978810076478e-05, P157=1.9606269539680884e-05, P158=2.6904608021098498e-05, P159=1.7897426210256424e-05, P160=1.6165319237730328e-05, P161=1.8799368103952865e-05, P162=1.8236434834607824e-05, P163=6.927126823303837e-06, P164=4.8971067494898806e-06, P165=4.668049528938566e-06, P166=9.775438696898985e-06, P167=1.7109274415508686e-05, P168=2.186179879630764e-05, P169=2.6367786113451288e-05, P170=2.633569614058862e-05, P171=2.5606921746735276e-05, P172=1.565714299380637e-05, P173=9.18204381173449e-06, P174=9.046797909833218e-05, P175=0.0003397690228665485, P176=6.885093026689132e-05, P177=3.644262119935912e-05, P178=1.3006412756098637e-05, P179=1.2389565719935532e-06),Row( P0=-442.4650016753677, P1=-17.054261719511256, P2=-60.95521040623539, P3=-14.84974566647118, P4=-20.977810122999042, P5=-8.4739328118891, P6=14.729869578944031, P7=15.197598450732288, P8=0.8724882779629788, P9=-4.819021970362418, P10=-14.337030606001932, P11=6.65453952390389, P12=-4.071169239418658, P13=6.34755219996122, P14=5.0251479123554885, P15=-5.54471455758212, P16=1.5346950511548203, P17=10.081711846471354, P18=8.083306056411793, P19=5.277292173706706, P20=6.589644696685132, P21=7.9768277137850445, P22=-5.88042707265765, P23=0.6619082196838998, P24=10.387047257176302, P25=-0.7280505343965095, P26=-0.3272680739588077, P27=3.3719617587268846, P28=1.781166910076635, P29=4.482366231804922, P30=-0.044522385730943796, P31=4.745744789947369, P32=0.6894434549007677, P33=3.799224836265698, P34=1.7494072768016067, P35=4.063051985587752, P36=-1.1513761490803258, P37=-0.4070872578959311, P38=-1.2156281080387876, P39=1.738176443540869, P40=0.6838360649271964, P41=0.5337319487428921, P42=0.3923920327888917, P43=0.41609939851154637, P44=0.44578522769446094, P45=0.48812375531081803, P46=0.5745124760369464, P47=0.5724370840862356, P48=0.46741277663463227, P49=0.4292410826361872, P50=0.5006394395000142, P51=0.6122517040201195, P52=0.0003322095876333525, P53=6.152073439528637e-05, P54=7.525184913068704e-06, P55=1.26018079230862e-06, P56=4.3626289180318015e-06, P57=7.546414251306632e-06, P58=4.609135575118625e-06, P59=5.221519352938413e-06, P60=2.136168256449657e-05, P61=4.901790649871996e-05, P62=7.2939815810192e-05, P63=8.618027052721573e-05, P64=5.351919453546114e-05, P65=8.176123688741323e-05, P66=0.00013646632149035129, P67=0.00016790986292551636, P68=9.445023125260454e-05, P69=0.00010607955077056959, P70=0.00010615982065366396, P71=5.167386362934125e-05, P72=4.890575518203107e-05, P73=7.236480404486147e-05, P74=0.00021295878238011503, P75=0.00044982796113718083, P76=0.0007608502308364642, P77=0.0015140851601446966, P78=0.0019245134947743006, P79=0.0032242861261369844, P80=0.009101041018668012, P81=0.029855601860359352, P82=0.1406770418920794, P83=0.15885219601952602, P84=0.10249805533521926, P85=0.1975803077016853, P86=0.1471771591604905, P87=0.151830219196573, P88=0.5991512372023415, P89=1.9874324748899874, P90=1.807218332749651, P91=1.3047778960798952, P92=3.1432226434103048, P93=3.9775370449877596, P94=7.089887021128851, P95=8.6809070406655, P96=2.731358796399322, P97=1.1185007088944208, P98=0.38509156100300257, P99=0.08224112268134089, P100=0.08283053100647163, P101=0.23683668478547493, P102=0.16686922756966432, P103=0.1769357168083939, P104=0.29387919600441315, P105=0.14431925880668062, P106=0.15345411704112083, P107=0.20803971452618816, P108=0.10498944843754543, P109=0.10870232378696977, P110=0.4358466946741541, P111=1.0727565246405137, P112=1.9714529180924865, P113=0.7093051082069938, P114=0.2470111903425182, P115=0.34643912979218283, P116=0.5715882960400046, P117=0.43743714194213446, P118=0.250778486251684, P119=0.13783419210913783, P120=0.07957121653288743, P121=0.08709294837362197, P122=0.18514076392899673, P123=0.3036021782000902, P124=0.4905067028868362, P125=0.847097068316286, P126=0.4611298304535298, P127=0.22507075489682082, P128=0.4765037375932291, P129=0.19100415910150043, P130=0.038105071289518885, P131=0.040273855091428445, P132=0.0871237064680006, P133=0.04184567981390759, P134=0.057552019019582505, P135=0.06560292190222328, P136=0.052562290613556495, P137=0.11993131302010199, P138=0.05314526746121825, P139=0.04084146269457562, P140=0.13741818602900507, P141=0.153169585893183, P142=0.08955558515269367, P143=0.04548979448968557, P144=0.05632925581763058, P145=0.013985605502857386, P146=0.023666163619667217, P147=0.05389992971353452, P148=0.030831286610068082, P149=0.014820138273044488, P150=0.014537876921721994, P151=0.003094840945814304, P152=0.002110013529898161, P153=0.0014938195194813537, P154=0.0008515816348702919, P155=0.0005306292101386991, P156=0.0003879918309844451, P157=0.0005189895016425873, P158=0.000676609497297292, P159=0.0006939307664218907, P160=0.00068699736209613, P161=0.0006439891029368007, P162=0.0007281613265195449, P163=0.0006553155352766663, P164=0.0006040294633148063, P165=0.0008889924707561401, P166=0.0010636194747243853, P167=0.001188098910489492, P168=0.0022512834420883015, P169=0.0017525928840432512, P170=0.0010871656210935908, P171=0.0010472086247205544, P172=0.0005533338154987589, P173=0.000491896851025932, P174=0.0005009817312716012, P175=0.0004951225014657698, P176=0.0007692946946070765, P177=0.0009155871246778144, P178=0.0005819852322161965, P179=6.154503280542656e-05),Row( P0=-594.9024863045983, P1=17.194995100687766, P2=-32.98131982375853, P3=-10.080569847711121, P4=-26.443132204980685, P5=-13.5743975745686, P6=-4.296037763020284, P7=-7.4749082656333625, P8=-4.388711095349035, P9=4.175615274445357, P10=8.197702751322852, P11=15.903427145383777, P12=4.432970887129629, P13=-0.5798285655131822, P14=-5.94686218138737, P15=-2.8210425653435602, P16=-2.625869483965946, P17=6.0110720866526375, P18=8.399702351972678, P19=3.2522042529754804, P20=6.115271459077319, P21=2.4617929328324184, P22=4.4567400124975896, P23=-1.3341401205870422, P24=-3.830382863749987, P25=-0.8977006049596995, P26=-0.31721625593437797, P27=2.4150892769236503, P28=-2.451295407126403, P29=1.734818607000335, P30=0.20950147323690219, P31=1.9307125024525387, P32=-0.0700907094264441, P33=1.8822835001780973, P34=-1.7753267092163023, P35=2.0384771094778587, P36=-2.6487274003910746, P37=0.45030404221763876, P38=-0.8782876159727884, P39=1.3332478677414623, P40=0.4931538227438038, P41=0.4427776889651751, P42=0.45353573572911593, P43=0.45084650442922486, P44=0.46533892982562247, P45=0.5355595339790515, P46=0.526526231100133, P47=0.5852089724881937, P48=0.6402787867596191, P49=0.6515631954844502, P50=0.6024166638611217, P51=0.5563167196470974, P52=5.0752215194740036e-05, P53=1.5028671236094394e-05, P54=4.975888192601001e-06, P55=6.750180463300717e-06, P56=3.0266354362958903e-05, P57=8.510518681023097e-05, P58=2.9765210847571187e-05, P59=3.5168125914006644e-06, P60=6.27612839704526e-06, P61=1.0514867679188869e-05, P62=3.7738251328005394e-05, P63=6.555488554389549e-05, P64=3.8386226004088994e-05, P65=0.00013675893601437295, P66=0.0005402773183233781, P67=0.0009742251869275741, P68=0.0006754498447592655, P69=0.0011254646674309606, P70=0.0017592537148002328, P71=0.0014316345997963727, P72=0.002462721694746215, P73=0.005791748959787483, P74=0.03607039382514503, P75=0.0773706265769286, P76=0.08320572336660682, P77=0.13245136958403558, P78=0.1605818224865664, P79=0.14972025926081284, P80=0.06430519795567877, P81=0.03548505952127248, P82=0.04082032674969231, P83=0.012741009825171583, P84=0.001195657977622053, P85=0.00031752643265130074, P86=0.0001677730894197769, P87=0.0001156805569440595, P88=0.0002806411139787762, P89=0.0002476094464725286, P90=0.0003429016372502857, P91=0.008384627127788292, P92=0.14802007680144538, P93=0.3276067863353949, P94=0.11283067793656945, P95=0.05840444911549524, P96=0.05536781736927391, P97=0.07008163068517359, P98=0.1497279348296185, P99=0.24902173004751293, P100=0.14335360279465548, P101=0.043995286598983004, P102=0.0015596255201249542, P103=0.0009712816729369865, P104=0.002480596660336769, P105=0.006827286512888324, P106=0.002714826839943279, P107=0.0011969066682959242, P108=0.001538216256836939, P109=0.00560398194800595, P110=0.0034339597529723202, P111=0.002736533843753643, P112=0.0010539766747877308, P113=0.0024873661882337168, P114=0.012157046923175888, P115=0.003399526551428738, P116=0.00023805923510456447, P117=0.0004061334213847924, P118=0.001194455382678787, P119=0.0005436529733581478, P120=0.0013483004047210533, P121=0.002099741332738684, P122=0.0008024622965313367, P123=0.0005529314266705158, P124=0.0003598010530823782, P125=0.0009630615109125983, P126=0.002890865517734163, P127=0.0018176813056463943, P128=0.000281916937958958, P129=0.00035944100396111724, P130=0.0004469755584385968, P131=0.001188155965886588, P132=0.0018610672319598325, P133=0.000867199422750105, P134=0.0015595193452979392, P135=0.0013101833199607314, P136=0.0021856468288139045, P137=0.003185539791516411, P138=0.002209295695920964, P139=0.0016767740842093919, P140=0.002773839077563143, P141=0.001963314445086713, P142=0.0017969880833864434, P143=0.0007753380030023713, P144=0.0005462693168438861, P145=0.0005174234910041148, P146=0.0005372898780965766, P147=0.00045326395334412665, P148=0.00043582716584866365, P149=0.0003380826986925461, P150=0.00025716925928531605, P151=0.00022902539685699805, P152=0.00017529681371775704, P153=0.00014951810792558304, P154=0.00017321269735296878, P155=0.00027602781897624563, P156=0.00025073785352197706, P157=0.00018430066453444072, P158=0.00015088733382740403, P159=8.739516095224926e-05, P160=0.00012371593974720558, P161=0.00012113340982595309, P162=9.630403767002432e-05, P163=8.81589907003679e-05, P164=0.00013117619455431463, P165=0.00010786945406375875, P166=0.00011060793945512993, P167=0.0001064897253122775, P168=0.00016968176704388008, P169=0.0001438956191176138, P170=6.585717316006456e-05, P171=3.944741423798838e-05, P172=2.796357524000691e-05, P173=2.795837862067633e-05, P174=3.621334566560722e-05, P175=4.837813613616539e-05, P176=4.60732077367785e-05, P177=4.0224900621350426e-05, P178=2.1773321256369542e-05, P179=2.8582039606194343e-06),Row( P0=-606.8532056083664, P1=19.10185359029287, P2=-60.711829003096895, P3=4.123836315900726, P4=-29.028009714376914, P5=-24.429665167628507, P6=-1.1735625329398336, P7=-8.4991844576992, P8=-8.285619445990429, P9=-12.72768480178132, P10=-8.111260148510814, P11=9.065168150853342, P12=-7.6222195498188485, P13=10.021051057895399, P14=6.444752922106321, P15=8.65701602540141, P16=12.859949332789952, P17=7.871613165524879, P18=11.535327783275017, P19=4.688254661833043, P20=1.3618532868852562, P21=5.061918404121089, P22=1.9454480076942833, P23=-0.8722916658766899, P24=1.4746299029412069, P25=-3.6796415085804437, P26=-0.806974754753122, P27=2.4249489466500904, P28=3.2483421239076953, P29=4.774162622061273, P30=1.0621210674288777, P31=1.9847505827377192, P32=3.563350588857868, P33=5.863321424960494, P34=-0.04820594282319727, P35=-1.687876417267778, P36=-2.2498935274758085, P37=2.1010416042474778, P38=-1.552937173216948, P39=1.1847716820390606, P40=0.6333720052114743, P41=0.6289551833295796, P42=0.572516905795855, P43=0.48811576632172987, P44=0.47512376657809596, P45=0.5150318475922532, P46=0.505576558614382, P47=0.5164274538138023, P48=0.5556303457971408, P49=0.5505187142551551, P50=0.5860495898326992, P51=0.6058676757846252, P52=8.153302973009971e-05, P53=4.473801269688531e-06, P54=9.18741619163597e-07, P55=1.0133718492820763e-07, P56=4.0058636216289926e-07, P57=6.481387613696003e-07, P58=2.3049597121192394e-07, P59=5.3004024875140396e-08, P60=4.3255237334171704e-07, P61=2.101391795432944e-06, P62=6.658592191133532e-06, P63=2.756734337283918e-05, P64=0.00022241032446613767, P65=0.001609055538119772, P66=0.004458299625981155, P67=0.009445638345483552, P68=0.009945858678997528, P69=0.014313155920650281, P70=0.0196918264647664, P71=0.006711238192088122, P72=0.003774420417255386, P73=0.0062278243595929724, P74=0.011041218634925845, P75=0.018831889257062005, P76=0.024571746926728755, P77=0.04422162292644773, P78=0.10339734343347903, P79=0.07403861091219587, P80=0.012602916462807855, P81=0.002689103812403247, P82=0.004389351994556164, P83=0.007732174253616402, P84=0.01875427327437901, P85=0.02817186877882933, P86=0.007227823827350454, P87=0.0028248239474771555, P88=0.002908863624178771, P89=0.0023387742918697073, P90=0.004707403500578722, P91=0.006682716103103436, P92=0.012303771722107485, P93=0.026573010501115008, P94=0.022314970608289564, P95=0.019701589405289537, P96=0.05279885915899515, P97=0.08801833339334662, P98=0.035802180933169815, P99=0.05327738408497139, P100=0.011543727649927555, P101=0.008114575592113104, P102=0.00623145861563328, P103=0.0026629203220819987, P104=0.0010756735017481871, P105=0.0024882009602835826, P106=0.012075274181308914, P107=0.014430428459957385, P108=0.03127139880032629, P109=0.013516642645872493, P110=0.0012845966283955453, P111=0.00044500658447969995, P112=0.0037320116622509216, P113=0.010657821451570543, P114=0.00652321132719816, P115=0.011127100051140914, P116=0.0030825284624206614, P117=0.0012567420059927253, P118=0.011464629685850899, P119=0.019180991154973268, P120=0.007328294637139487, P121=0.008500757333211477, P122=0.004148394853527488, P123=0.012638451276910668, P124=0.0029341219491670852, P125=0.001670482735095541, P126=0.0011861918827992264, P127=0.007169864980083358, P128=0.0019060044460337423, P129=0.0062813438881504135, P130=0.005221084501953445, P131=0.004875928495990279, P132=0.0035484187393862257, P133=0.004702206289262869, P134=0.002886279220164161, P135=0.0016211908884850056, P136=0.0015305160626370377, P137=0.004074023708086679, P138=0.0008667405597203137, P139=0.0016532471353100041, P140=0.0012969819714686727, P141=0.00012581402695312755, P142=0.00033084191354828445, P143=0.00017723967815762165, P144=0.00042264696369047073, P145=0.0002975770018033173, P146=0.0013503232024558295, P147=0.0013312890833593839, P148=0.002043802680907431, P149=0.002522180600659395, P150=0.0009095745279467122, P151=0.0005587184200097611, P152=0.00017052648707929085, P153=8.641361310362287e-05, P154=3.0111222967102067e-05, P155=3.923776608241649e-06, P156=4.405337535953752e-06, P157=7.363653013447114e-06, P158=9.848442498330652e-06, P159=1.1805588928480742e-05, P160=1.0720708907917854e-05, P161=6.7507704313808106e-06, P162=5.496960719730453e-06, P163=6.7439385370444144e-06, P164=5.0417115827925884e-06, P165=5.550525464135526e-06, P166=5.543751501578168e-06, P167=5.990350133441669e-06, P168=5.717739574235417e-06, P169=9.82924976626343e-06, P170=1.238994740870714e-05, P171=7.997543068087554e-06, P172=7.446994400841283e-06, P173=6.710745333012745e-06, P174=2.8978609491265306e-06, P175=6.27944389815137e-06, P176=1.3266806306238524e-05, P177=2.293604841649644e-05, P178=6.9157074055211686e-06, P179=4.3894265742125144e-07),Row( P0=-750.2908393519249, P1=43.92431649554667, P2=-14.834592708020681, P3=-3.125250028680032, P4=-17.785517212206194, P5=-18.162801619459362, P6=-3.776111404246075, P7=0.47325769614834434, P8=-10.576013054681308, P9=-10.161860037030685, P10=-5.116367363714254, P11=-3.064789325904483, P12=-8.4700683738826, P13=-5.184288701912733, P14=-1.0678006139773812, P15=5.494854127971916, P16=4.665092847464113, P17=11.560888613731864, P18=11.86225886945271, P19=17.401204343203155, P20=12.484588727994959, P21=12.489962063773174, P22=7.548996087431197, P23=2.875089365953308, P24=2.6947712893279476, P25=-2.265639198558408, P26=-1.788059330310985, P27=2.9239493650264796, P28=0.9137778084247606, P29=3.227707402777107, P30=-0.8203958418639146, P31=0.35578867895818034, P32=-1.729509825802574, P33=0.5346581398863929, P34=-2.980143523334785, P35=-2.1352489048126975, P36=-6.220706262225834, P37=-0.9279292287541897, P38=-1.7376147160918556, P39=2.1159555074873295, P40=0.6752152985173894, P41=0.6161499430530327, P42=0.551679154813207, P43=0.5321033922780797, P44=0.5270165509111696, P45=0.5425305294123949, P46=0.5744271666698376, P47=0.6621298072112469, P48=0.7240208338362927, P49=0.7075780469444661, P50=0.6543656770746838, P51=0.6333834886088935, P52=3.6556205574602494e-05, P53=3.2525777497354335e-06, P54=1.06831916491361e-06, P55=2.0794219491829114e-07, P56=1.7750060679609172e-07, P57=2.549002989396244e-07, P58=7.394620086936667e-08, P59=5.091537333835561e-08, P60=9.080724938207843e-07, P61=1.0827799843468721e-05, P62=0.0001786975217887064, P63=0.0008514253835773407, P64=0.002104166687235061, P65=0.0032173873395755616, P66=0.0021143979508346034, P67=0.004985476664618647, P68=0.005750509285936337, P69=0.0016675889248863388, P70=0.00014440950185147974, P71=2.5485844639175612e-05, P72=5.5653076219842086e-06, P73=6.0807764046274285e-06, P74=1.1271208649965425e-05, P75=3.193915460275871e-05, P76=0.000165526442150859, P77=0.0006351605118608703, P78=0.0009192160846651946, P79=0.0007115442315561811, P80=0.00016495213437116895, P81=6.35295609440322e-05, P82=0.00018627339926617994, P83=0.0008488030342818562, P84=0.0025223414227904498, P85=0.00853378341480738, P86=0.003084633705739338, P87=0.00013421566826976445, P88=8.850700526030071e-05, P89=0.00016619005168176898, P90=0.00013259553575970323, P91=0.00011672545305205284, P92=6.109343607315132e-05, P93=2.072229888619409e-05, P94=1.591010546079413e-05, P95=7.685331242983358e-05, P96=0.00019189771681102577, P97=0.00010986675050237012, P98=0.0001054908000799318, P99=2.522394008833729e-05, P100=1.3179259960831214e-05, P101=1.8720443482607012e-05, P102=2.802922097512846e-05, P103=1.8259063273933716e-05, P104=1.841175588351483e-05, P105=1.8859799378826418e-05, P106=6.25983572466317e-05, P107=4.190119780575183e-05, P108=1.4293190158384377e-05, P109=6.116505814671366e-06, P110=3.2836954012429788e-06, P111=1.948954975491389e-06, P112=7.012869223045363e-06, P113=8.277013583355407e-06, P114=4.3926558048750414e-06, P115=3.1797630631541625e-06, P116=4.3976525616740405e-06, P117=6.230410935999593e-06, P118=7.44635454143367e-06, P119=6.980572094305661e-06, P120=6.467735948686466e-06, P121=1.1486089146375706e-05, P122=2.366244919054044e-05, P123=1.742211741837552e-05, P124=9.443288678621678e-06, P125=1.2601403522279102e-05, P126=1.0019965535189084e-05, P127=5.528895049109809e-06, P128=5.946900208839609e-06, P129=8.852330647310815e-06, P130=4.407725346718929e-06, P131=6.980944373009711e-06, P132=6.984772879911432e-06, P133=7.729768741057707e-06, P134=1.6879584897845813e-05, P135=2.1841931874622907e-05, P136=1.2649398224071186e-05, P137=1.808670811956425e-05, P138=1.2277616204946287e-05, P139=1.3814513230658796e-05, P140=9.208340964403156e-06, P141=6.585205234228897e-06, P142=3.993533462263859e-06, P143=3.936731532473915e-06, P144=3.485561119104139e-06, P145=3.8842037483761365e-06, P146=3.836546948776227e-06, P147=4.597711604037912e-06, P148=8.413271389385576e-06, P149=9.543645629385593e-06, P150=6.614982207776447e-06, P151=2.998625992343619e-06, P152=1.8631421269720592e-06, P153=2.0769924990344787e-06, P154=3.5776153588640476e-06, P155=7.1889465028789765e-06, P156=1.626336888566453e-05, P157=1.744611083419262e-05, P158=1.7977411622198763e-05, P159=1.6292343810346565e-05, P160=1.9323470218263196e-05, P161=1.6146772347673312e-05, P162=9.344838475662207e-06, P163=5.968846150311497e-06, P164=6.610808810755754e-06, P165=5.892922099200013e-06, P166=1.875163356916654e-05, P167=2.6760063211357723e-05, P168=1.188913264331251e-05, P169=9.659186095582681e-06, P170=3.062914308364275e-05, P171=4.5612113365465435e-05, P172=3.381683139489003e-05, P173=2.6166108887424525e-05, P174=1.693559086143063e-05, P175=2.3021074518503607e-05, P176=1.9468700492416436e-05, P177=1.5087676646242006e-05, P178=4.834670053863704e-06, P179=7.534132268915461e-07),Row( P0=-502.60867401429635, P1=-10.277943010661453, P2=-59.31401514776261, P3=-5.361942399959113, P4=-23.036786487641766, P5=-16.114374589238544, P6=5.443516612418126, P7=14.19442646381394, P8=11.246641915768132, P9=9.637993059688336, P10=-8.52379648331084, P11=1.486759479056984, P12=-11.432007500747467, P13=9.094145739633513, P14=11.26094234508451, P15=0.7236251370835687, P16=-2.3218620081537917, P17=0.38516482110000366, P18=12.846823919511216, P19=4.003803923685802, P20=2.408198859958119, P21=9.23142012884717, P22=6.367228505360732, P23=-0.9341903791351994, P24=-2.750708854244486, P25=0.8566342076853719, P26=6.730579474526122, P27=2.825639571251023, P28=-0.6046925714207569, P29=3.80174779580136, P30=0.9412694880649138, P31=0.8162504822554695, P32=3.7866909886494478, P33=4.026982752536088, P34=-2.3058278490870103, P35=5.427969219938446, P36=1.838378076246322, P37=0.18395540739999394, P38=-1.8456441626967626, P39=3.3827992601196977, P40=0.629825782885163, P41=0.5867806756281854, P42=0.6242818087989959, P43=0.6380385664425895, P44=0.5563486442538601, P45=0.4583204889343379, P46=0.4493865977263688, P47=0.4860815016651328, P48=0.48563109718921227, P49=0.5746943736644079, P50=0.6601040953017961, P51=0.6509316100427094, P52=0.0035041469231921005, P53=0.0003018434126077045, P54=7.215973214972549e-06, P55=1.5838745735714973e-06, P56=4.9001699000422184e-06, P57=6.358893522639259e-06, P58=1.733777431163034e-06, P59=8.67303851640237e-07, P60=5.226281723829906e-06, P61=1.1396011203023529e-05, P62=1.9889431772116574e-05, P63=2.0997109587978902e-05, P64=1.4777972955749482e-05, P65=4.0866224145627594e-05, P66=6.319896224272926e-05, P67=6.940668438894576e-05, P68=4.273296104360265e-05, P69=4.331608045890565e-05, P70=5.3621838439125135e-05, P71=5.954845890625217e-05, P72=0.00010627785464243935, P73=0.00023854043257213317, P74=0.0014503971364743446, P75=0.0034766093546667945, P76=0.003535966251579896, P77=0.006269150652407606, P78=0.011575430365380671, P79=0.025992649595533664, P80=0.0641599800799679, P81=0.06160605470018972, P82=0.053526393447152834, P83=0.09586973883826, P84=0.13353636664335966, P85=0.1364551212824065, P86=0.1306155420085415, P87=0.316228369211135, P88=1.517874702302546, P89=1.5724267193558161, P90=1.17845401601329, P91=1.5292560086497922, P92=0.6289644638269681, P93=0.11322632227756396, P94=0.023933454633632962, P95=0.06868985880981518, P96=0.05852630698586637, P97=0.0366197977893541, P98=0.023164676752564343, P99=0.029071927943340962, P100=0.06423354989777312, P101=0.10863169252545127, P102=0.0914651581044125, P103=0.028692577305500493, P104=0.018680689419369675, P105=0.042106041037046654, P106=0.13386384440044943, P107=0.1659171123759375, P108=0.705390184610453, P109=1.08547167492912, P110=1.1353776663764725, P111=0.7233957719958913, P112=0.25498262788640863, P113=0.2669922834714914, P114=0.09118538090738595, P115=0.02766615537606155, P116=0.038869404245076504, P117=0.04975948054933544, P118=0.05747186267870822, P119=0.05551683568054455, P120=0.16428181418754895, P121=1.0428155057598645, P122=0.602629854406412, P123=0.5045337399160904, P124=0.37393383564116456, P125=0.3582142889194831, P126=0.06771360108511777, P127=0.012058630691111787, P128=0.021702053575326497, P129=0.04322658438399152, P130=0.3799700458230288, P131=0.35285284739425377, P132=0.04206095922816302, P133=0.019936249792715143, P134=0.014584541441682782, P135=0.009246797073353174, P136=0.017873431998940308, P137=0.0636825056510765, P138=0.05211578507832614, P139=0.04694311755685656, P140=0.022916164062877536, P141=0.0071959480766957155, P142=0.006540325264580095, P143=0.011303115593623335, P144=0.009238437403498768, P145=0.007502908166142934, P146=0.009106259132544672, P147=0.01988978321503348, P148=0.021054782006193096, P149=0.008695054511939334, P150=0.005228887455647013, P151=0.008673836128652827, P152=0.0063796930881299865, P153=0.0005723716009034852, P154=0.00025620686477190174, P155=0.00036897436113492, P156=0.00017496518056207933, P157=0.000304877051362197, P158=0.0003814378269012088, P159=0.0003748459674402978, P160=0.0003078220880317121, P161=0.0003633775071859911, P162=0.0003712365386100761, P163=0.0004895799083671102, P164=0.000571967619167003, P165=0.00042283476165955446, P166=0.0003442806524812141, P167=0.0004771730018625949, P168=0.0004054561346764523, P169=0.000606843354257642, P170=0.0003289374290723407, P171=0.00016521820076444298, P172=5.684881980165597e-05, P173=5.1767836962814404e-05, P174=5.39642924834925e-05, P175=8.500619057640331e-05, P176=9.718561616436634e-05, P177=0.00010521797713935644, P178=6.948726585155563e-05, P179=8.562196950871543e-06),Row( P0=-425.92793205095944, P1=-19.14724485216539, P2=-69.74478353075301, P3=-8.080945744804202, P4=-17.811324406312185, P5=-0.9593347545000096, P6=14.62626265802824, P7=21.103608473262355, P8=-2.63529685304771, P9=-8.888187839157705, P10=-13.921507702702682, P11=10.7356281864289, P12=3.815026827139669, P13=9.192487162892446, P14=0.37373187843933975, P15=-11.924228774593864, P16=5.330805190577561, P17=11.617978250800789, P18=9.403244083071197, P19=2.854504604747627, P20=5.8518099607539655, P21=9.237882599260722, P22=-6.195147016741466, P23=0.865699960950523, P24=12.755487416857232, P25=-2.904347188096017, P26=-0.5597297130072627, P27=3.531229478220581, P28=1.381756863586303, P29=3.510011349029458, P30=1.3616279018333382, P31=3.9528177311462085, P32=0.20234012451423403, P33=3.994719407497002, P34=0.9258900524107311, P35=3.174498862102465, P36=1.4388095782224826, P37=1.1721141128151615, P38=-1.9880772470403654, P39=1.1119329200897126, P40=0.5839235953059856, P41=0.5742250533694807, P42=0.45673534895548507, P43=0.37208872221768263, P44=0.4153041682605007, P45=0.498188544193185, P46=0.5872213380636824, P47=0.6642578103096742, P48=0.49683798454533645, P49=0.39408120691363413, P50=0.43075760683449926, P51=0.5044622609196933, P52=0.0010006298252104812, P53=0.0002240586096520267, P54=2.316989521060801e-05, P55=2.569902225858671e-06, P56=6.3443843696956674e-06, P57=1.1782464405689006e-05, P58=5.446796737453899e-06, P59=5.763244250701898e-06, P60=2.1148831316014716e-05, P61=4.547655843267843e-05, P62=5.332666825896437e-05, P63=5.018475173001915e-05, P64=2.9406723208047535e-05, P65=3.426960216570951e-05, P66=8.177257723839924e-05, P67=0.00011317449461287104, P68=5.372332262235756e-05, P69=4.493943110673708e-05, P70=4.5116232135840945e-05, P71=2.4236834528515326e-05, P72=3.1173483704405456e-05, P73=3.934822404614111e-05, P74=0.00010638099919707029, P75=0.00014265981585858526, P76=0.00010718572504334532, P77=0.00021452898739884055, P78=0.0008119318361429907, P79=0.003267783367256673, P80=0.0078269308042432, P81=0.013680266971461928, P82=0.018518387256951055, P83=0.011999819013942484, P84=0.013603672939507322, P85=0.022769948106552466, P86=0.018836816394830678, P87=0.027945397206281277, P88=0.07352086328364993, P89=0.16279905004327969, P90=0.41375857170238456, P91=1.4149506871415087, P92=2.1221588988335114, P93=2.909354547701138, P94=6.573686235706875, P95=12.599875717923434, P96=11.167744220893855, P97=4.815622390381352, P98=4.421722945253399, P99=1.6437004943850768, P100=0.5993310353507142, P101=0.620631109680886, P102=0.21909955507523532, P103=0.10515609131676613, P104=0.029418313695626135, P105=0.013277068049657637, P106=0.10142575842106516, P107=0.5649970506593517, P108=0.09843786204445379, P109=0.015432270514997487, P110=0.021707954815519678, P111=0.2608882769373139, P112=4.7807621756754495, P113=7.147078026864347, P114=2.6323531190767704, P115=8.656222420681877, P116=6.231601385064164, P117=1.9321150168542893, P118=1.7045033896815733, P119=2.818070700463181, P120=0.7540585179187044, P121=0.30377077422160464, P122=0.23073498779264154, P123=0.3419678055508166, P124=1.6538759929595843, P125=1.96258066858399, P126=0.8173896358443841, P127=0.369343445160305, P128=0.6063430047527498, P129=0.4960061598054389, P130=0.20946573916418526, P131=0.07149166779111203, P132=0.0774495697474854, P133=0.0860332443632989, P134=0.08459236898578161, P135=0.10272307927870164, P136=0.35954963823758734, P137=0.4789065194536187, P138=0.18870894067991661, P139=0.06593880742435239, P140=0.05581831910664941, P141=0.06530047445581347, P142=0.06337706465104699, P143=0.07525697205827399, P144=0.10872543550949999, P145=0.05309222404129598, P146=0.08350147866761877, P147=0.09011035332859473, P148=0.06522675456041845, P149=0.02675258632832349, P150=0.03640884505896935, P151=0.011942411084981275, P152=0.003229384518747174, P153=0.0011354007316406233, P154=0.0007448228794737329, P155=0.0010009893580750005, P156=0.0011372342682349343, P157=0.001612191496244184, P158=0.0013223069525326277, P159=0.0012135993833251799, P160=0.0014451457686541146, P161=0.0016137636671119883, P162=0.0014569042818164922, P163=0.0011239050017875535, P164=0.0019148758921383655, P165=0.0023405640191655063, P166=0.002010150678977416, P167=0.0025695934619256234, P168=0.002816869506877834, P169=0.002232237441824651, P170=0.0016139249779486963, P171=0.0008860770648364779, P172=0.00044887185484834727, P173=0.0003417271347036787, P174=0.00034524999135953667, P175=0.0005774592571467191, P176=0.0006268428878250254, P177=0.0007208675982919, P178=0.00031546802120119164, P179=3.664089414177944e-05),Row( P0=-707.4042063658555, P1=54.45638045596135, P2=-27.97512078076621, P3=-8.974728932403286, P4=-16.935558158933397, P5=-20.21048518903838, P6=-3.355576482138135, P7=3.9318172961934503, P8=-11.831561532350051, P9=-12.418526878456415, P10=-2.305889304432092, P11=1.179656210969451, P12=-9.529985949130328, P13=-0.07172888823984416, P14=0.4880597885080633, P15=3.2196439500237717, P16=9.194536540758289, P17=11.52854127523511, P18=12.438452968714635, P19=16.772411731918805, P20=11.596083190537069, P21=11.152395705843388, P22=5.572150561516786, P23=5.347221089705305, P24=5.574794306594961, P25=1.5048429482711092, P26=-2.4563210539463047, P27=-1.864151933582961, P28=-0.6260144547861037, P29=2.296458250549512, P30=0.4727475040328717, P31=0.5534211468241559, P32=-1.5133673444671383, P33=-0.5327355698676774, P34=-2.628903496187008, P35=-1.090786516457952, P36=-5.44026731310575, P37=-0.6914603458543837, P38=-3.4243231692846288, P39=-0.8970646356801661, P40=0.6935113245841655, P41=0.5998087688486035, P42=0.4909629476833764, P43=0.4558469706661605, P44=0.4561147833613694, P45=0.5012171325616296, P46=0.5339493008944114, P47=0.5518551240983967, P48=0.5631281006101835, P49=0.5613456737744953, P50=0.6121773488281963, P51=0.6291108601037158, P52=7.66797167322339e-06, P53=1.4289061372922076e-05, P54=4.893706249375347e-06, P55=7.222181592046782e-07, P56=2.133991642912241e-07, P57=5.2948313598120784e-08, P58=3.1785701487213164e-08, P59=8.612532574656625e-08, P60=3.6193891132418506e-06, P61=8.58460544173702e-05, P62=0.0016945384565017424, P63=0.002861511887413867, P64=0.002433462516976853, P65=0.007609616683368816, P66=0.010858889936119977, P67=0.02172948182558947, P68=0.010639528419439513, P69=0.000635306405068538, P70=0.00010140020765230957, P71=2.8792761708129715e-05, P72=2.499818966249768e-05, P73=0.00016500170030110046, P74=0.0010309127129691559, P75=0.001173524001242211, P76=0.0018032118112808837, P77=0.0034295003198156486, P78=0.0024178754793509768, P79=0.004862455135518813, P80=0.00797642084406628, P81=0.006106246038715141, P82=0.005231170210336394, P83=0.007125718638604705, P84=0.008833662573488539, P85=0.008550707607742578, P86=0.0019490991991440022, P87=0.0006593504290775266, P88=0.0017966196475107494, P89=0.002125843244167583, P90=0.0008010934097068587, P91=0.0006428302840263983, P92=0.001069016454569815, P93=0.0007315741919591597, P94=0.0009406123664436625, P95=0.0066433112642663725, P96=0.009876126378395638, P97=0.005444304524513911, P98=0.0018935279131325313, P99=0.0006185037478626836, P100=0.00026334739496360147, P101=0.0003984759602594746, P102=0.00024733995613135504, P103=0.0002486342325741846, P104=0.011784722406381763, P105=0.006286218097329087, P106=0.0004679291116809983, P107=0.00010040447475470314, P108=2.7593569502697864e-05, P109=6.702462306285222e-05, P110=0.00013832253628921648, P111=7.297827607813478e-05, P112=0.0005023320548471011, P113=0.0002927466938644013, P114=8.854900607543368e-05, P115=0.00014537534831153093, P116=8.87084649418257e-05, P117=0.00026643231547519446, P118=0.0007896309909847919, P119=0.0003890731127158219, P120=3.165794307885485e-05, P121=8.05235297731423e-05, P122=0.0002875883487594338, P123=0.00021209697479760385, P124=2.3340897382124657e-05, P125=7.108615479056428e-05, P126=0.00010162745373912484, P127=2.24847102980689e-05, P128=6.443540607338197e-05, P129=6.559985871569167e-05, P130=1.9891172965142175e-05, P131=6.027304929802148e-05, P132=3.1720791145178254e-05, P133=5.01699654803765e-05, P134=6.631177542826593e-05, P135=1.3363385607536001e-05, P136=1.462434923146044e-05, P137=8.070859041073795e-06, P138=5.179221896006461e-06, P139=5.572451989471474e-06, P140=4.732695471613771e-06, P141=2.4223791288539134e-06, P142=1.5793278800435174e-06, P143=1.3560294233582997e-06, P144=1.6465161795368554e-06, P145=2.7833577232687394e-06, P146=5.333416636060623e-06, P147=7.800266010303721e-06, P148=8.217341186860966e-06, P149=4.459279383994416e-06, P150=4.152910388909616e-06, P151=6.011277534615404e-06, P152=6.004889582122504e-06, P153=2.758772086033669e-06, P154=1.9048339912076801e-06, P155=2.312703481417567e-06, P156=6.231046045691202e-06, P157=1.888420366624155e-05, P158=2.6497478877105593e-05, P159=7.867870545889418e-05, P160=0.00011117276830416164, P161=9.607550471393341e-05, P162=3.587871466714567e-05, P163=4.5459033719210366e-05, P164=4.3191999541541644e-05, P165=2.585689468248271e-05, P166=1.325593722809442e-05, P167=1.6617431951912305e-05, P168=3.8816216425872486e-05, P169=6.13231953298498e-05, P170=9.646487937385463e-05, P171=0.00013332390158757063, P172=9.55472489435343e-05, P173=3.8436587779891825e-05, P174=9.983099919619852e-05, P175=0.0001727230378785117, P176=0.00020260057615888703, P177=0.00012704762366855284, P178=4.0263357524809736e-05, P179=2.8817050784203023e-06),Row( P0=-709.032502201072, P1=50.91219306343749, P2=-24.701145185610674, P3=-11.4858953970966, P4=-22.29302630212166, P5=-15.614680530303785, P6=-9.566788821608384, P7=-3.9612417136746347, P8=-10.583295489009696, P9=-13.081236293556225, P10=-9.973536413613655, P11=-5.165002161473306, P12=-7.267615017577112, P13=-3.3352734311835412, P14=-1.4504890588462949, P15=7.643160078962644, P16=6.884382652676381, P17=8.087819033228378, P18=11.210403380392533, P19=10.689686497227637, P20=1.784603478041363, P21=3.5908482735158813, P22=-5.325500091565625, P23=-1.577206603405707, P24=-0.3024544491364417, P25=2.9489242787217744, P26=-0.5328327410525814, P27=0.8500519947737749, P28=-4.450981389348941, P29=0.48118846777425717, P30=0.0387627693448812, P31=1.210370971909534, P32=-2.006609526795183, P33=1.948059354323198, P34=-0.9097976225187061, P35=3.4714275812586863, P36=-2.342316674305155, P37=-1.7685369644683377, P38=-6.590081641427172, P39=-1.1463626971771859, P40=0.6821455098772073, P41=0.5442284640031819, P42=0.5540949103033938, P43=0.57405703854124, P44=0.5618050723763008, P45=0.5623966165871032, P46=0.5457206326801382, P47=0.5837595931884658, P48=0.5724601509396111, P49=0.5659558361487975, P50=0.6447546635599215, P51=0.7346479197956289, P52=2.981221266310145e-07, P53=1.1143322539336512e-06, P54=1.3260629638899393e-06, P55=8.191422029570814e-07, P56=1.1297211760597644e-06, P57=1.6282982395404204e-06, P58=7.992024752875031e-07, P59=1.177762305873726e-06, P60=7.840590079151929e-06, P61=2.140953789563722e-05, P62=5.944503489880437e-05, P63=0.00015931931246450335, P64=0.00015683169888561604, P65=0.0007566215593803759, P66=0.004307742176048197, P67=0.00659794907834259, P68=0.0017362902408589382, P69=0.0005768999953213933, P70=0.0011603805403366803, P71=0.0008939012237318554, P72=0.00024145142356965754, P73=6.370050251761207e-05, P74=7.44152421533231e-05, P75=0.0001588397638533849, P76=0.0002198810702197806, P77=0.00038040573309698183, P78=0.0007796427149816243, P79=0.0016049016194292927, P80=0.0016300110310092147, P81=0.0022505328337574285, P82=0.0027343726345165184, P83=0.0011674586365266598, P84=0.00036543512062463246, P85=0.0002739782615448056, P86=0.0003776117215546926, P87=0.0009675949763997546, P88=0.0024657065063588813, P89=0.002799938365500792, P90=0.0024925862322491194, P91=0.0014922977277674386, P92=0.0011929457617132347, P93=0.0010696820665874608, P94=0.0010338484608892057, P95=0.0008448674171853919, P96=0.0008039711131982668, P97=0.00025425939619751697, P98=6.469527516667103e-05, P99=0.00012432681010088363, P100=0.00016695740331702802, P101=0.0001494361362279052, P102=0.0001493785316250792, P103=0.00020707054082705359, P104=0.00016021955126008485, P105=0.00011281252462636319, P106=1.364179848336225e-05, P107=3.829180083584208e-05, P108=6.256451877719501e-05, P109=6.831318266731988e-05, P110=2.9167652882014826e-05, P111=3.557173899802635e-05, P112=4.246629624693775e-05, P113=4.991557891029409e-05, P114=5.677276404794451e-05, P115=2.432345518951757e-05, P116=2.543062255718791e-05, P117=3.4430224544954584e-05, P118=4.353792966355085e-05, P119=4.180181049990299e-05, P120=3.1331547304375975e-05, P121=4.0897119314600577e-05, P122=5.8360601591913164e-05, P123=3.056444631823113e-05, P124=1.4649021432721263e-05, P125=1.2699359004810673e-05, P126=1.6656306745261503e-05, P127=9.17306970665557e-06, P128=9.348709897270684e-06, P129=1.2028951357479163e-05, P130=1.3241918791691992e-05, P131=1.2671001617482044e-05, P132=1.5011299672106578e-05, P133=1.3833815937397482e-05, P134=9.953217532246296e-06, P135=1.6401495133015254e-05, P136=1.600659656654653e-05, P137=2.196028570159489e-05, P138=2.4162327436732e-05, P139=1.7099042308934046e-05, P140=2.546424403155157e-05, P141=2.28399031818822e-05, P142=2.8925373578229385e-05, P143=7.439920563259833e-06, P144=5.951649916845217e-06, P145=5.3958684454180456e-06, P146=8.266681171529642e-06, P147=2.273155509397915e-05, P148=2.544581733383252e-05, P149=3.047417451907156e-05, P150=2.2527494587479465e-05, P151=1.694004752527506e-05, P152=1.267207407078285e-05, P153=7.142169994360027e-06, P154=1.1537701217616361e-05, P155=1.1941968863970222e-05, P156=9.805317424731999e-06, P157=9.588621702254265e-06, P158=1.0011354157118318e-05, P159=1.4851490263881266e-05, P160=2.5736117103531325e-05, P161=1.4137378479187148e-05, P162=1.543262513488713e-05, P163=1.4728777465897507e-05, P164=2.0350814890772583e-05, P165=1.3871519663718849e-05, P166=9.119869562621106e-06, P167=9.95267575745852e-06, P168=5.749736961456911e-06, P169=4.0146781520479355e-06, P170=4.80996656845898e-06, P171=5.731730118277997e-06, P172=6.424394330316697e-06, P173=3.415588104191861e-06, P174=3.7720413970967445e-06, P175=5.1329317417486715e-06, P176=4.706504856073702e-06, P177=4.965774483796447e-06, P178=2.374010971430193e-06, P179=2.1016487365073477e-07),Row( P0=-501.82956930655246, P1=-14.879621600555883, P2=-60.73852014230909, P3=-6.693073086389725, P4=-24.603516614079012, P5=-16.248967255918437, P6=11.735871296707439, P7=15.429706381726074, P8=14.314401527337328, P9=6.5937407494792835, P10=-5.494964621310236, P11=1.382091915331803, P12=-10.291764864027474, P13=13.18365305263859, P14=13.000846046209045, P15=3.4069172898325277, P16=-5.506625564584989, P17=1.0352242739585866, P18=11.502715119373669, P19=7.412080982111269, P20=4.433524145403438, P21=7.686585409979345, P22=5.292487616359617, P23=3.626195627948565, P24=1.9384299950315533, P25=-1.0586231073108778, P26=5.187756730145979, P27=6.454265088423967, P28=0.15534500300572307, P29=2.2553763179242146, P30=4.147289244844482, P31=2.3069063318941483, P32=0.5315701162821654, P33=6.7306123420267685, P34=-0.6325520532623228, P35=2.1410561657845624, P36=1.990013168957035, P37=5.589818592460458, P38=-1.6488976991765756, P39=1.2286645530055387, P40=0.6556617532317863, P41=0.6130605076680107, P42=0.6475031840002508, P43=0.5792860533169825, P44=0.5274203189313877, P45=0.5240757009720889, P46=0.4818800369903077, P47=0.46938142168473146, P48=0.47883359723179053, P49=0.528330542922765, P50=0.5468638778862764, P51=0.6074947151843053, P52=0.006730118408249441, P53=0.0002664708213620403, P54=5.288426111810768e-06, P55=5.879302341866542e-07, P56=1.3804052517010195e-06, P57=2.5015886971235917e-06, P58=1.8105310764567917e-06, P59=1.203337549846533e-06, P60=2.3262842985799987e-06, P61=6.806505648167296e-06, P62=1.8908174582310393e-05, P63=2.4807918181253358e-05, P64=1.3175183950994382e-05, P65=1.9200953807616434e-05, P66=4.3241149964363615e-05, P67=4.3186246109858034e-05, P68=2.1160432652209053e-05, P69=1.8345010605729024e-05, P70=3.3553877831352e-05, P71=0.0007198046257167425, P72=0.0015392623086952538, P73=0.0015639786181847828, P74=0.008274820250500404, P75=0.008812661579683502, P76=0.005722403086676994, P77=0.007330071541294405, P78=0.00799756655684335, P79=0.007986126273009164, P80=0.013251868668669804, P81=0.019514887907735857, P82=0.03524215217579447, P83=0.04367292119838137, P84=0.08249864025385274, P85=0.29587054624153053, P86=0.3268433692771054, P87=0.47366909356741754, P88=0.9457371915405083, P89=0.5507084574911153, P90=0.4796809848819291, P91=1.769078885124773, P92=2.1679688942385895, P93=0.42858493594115804, P94=0.08744056237917051, P95=0.05633971503003403, P96=0.03749747429117286, P97=0.037321546396933664, P98=0.052859481169022185, P99=0.0751592558754286, P100=0.07954583862903623, P101=0.1767304352821211, P102=0.22341940851070535, P103=0.20473920102510523, P104=0.12192517489600235, P105=0.08422542199591897, P106=0.16775679221487738, P107=0.5880126891854688, P108=0.5218274140706589, P109=0.6508086247230391, P110=0.1819717025824029, P111=0.12782420263817348, P112=0.2420781267636494, P113=0.5454432330792574, P114=0.17293608642336608, P115=0.06782962881498004, P116=0.0554601186173276, P117=0.037164830136743834, P118=0.07377700788455302, P119=0.4125575659082099, P120=0.7070874260622048, P121=0.8637892295950141, P122=0.38050267467126997, P123=0.0933057887638554, P124=0.05413499561809956, P125=0.45437764424080096, P126=0.2619190491297305, P127=0.051590653108085145, P128=0.10029602503916615, P129=0.12120919767970333, P130=0.5081065325757576, P131=0.1971721145997666, P132=0.012304013597789642, P133=0.009937003229055726, P134=0.06133343865369323, P135=0.08190311144977637, P136=0.05274177733000963, P137=0.1231587772698549, P138=0.020652221947091494, P139=0.006017272713437169, P140=0.010142996685442935, P141=0.019959671065480827, P142=0.022594833826012444, P143=0.016030096265435452, P144=0.004184328073918646, P145=0.014832958732603315, P146=0.028891770637152155, P147=0.03084362683947501, P148=0.009872231577716255, P149=0.0075949875335761955, P150=0.00572489131807527, P151=0.011656227676899798, P152=0.004249591399646446, P153=0.0020319159033817505, P154=0.00102237614411424, P155=0.0004189199599837507, P156=0.0003193860002508241, P157=0.0001165511231928126, P158=8.518498125855861e-05, P159=9.149360785705623e-05, P160=0.00015594781846907428, P161=0.00018393919221464255, P162=0.0001457518605909919, P163=0.00010123454128698496, P164=0.00015158474187135113, P165=0.00013390415855554072, P166=0.00014046699764995994, P167=0.00015268469234648536, P168=0.00028073566259122955, P169=0.00047100257406919516, P170=0.0005373901095832951, P171=0.0004537353841368683, P172=0.00026272508614498866, P173=0.00019141558328445685, P174=0.00018674814065694468, P175=0.00047134285998813244, P176=0.00074547316140583, P177=0.0010514950000005295, P178=0.0006141384570088698, P179=8.23699420766377e-05),Row( P0=-754.3891989874618, P1=57.40611748047159, P2=-19.311193323163465, P3=-10.351502868962582, P4=-13.69260589539148, P5=-13.812677485257653, P6=-6.0587897449870125, P7=-0.6568178984245633, P8=-7.647640177077524, P9=-9.87185991769832, P10=-6.665950368467331, P11=0.24776645450497364, P12=-4.553553095206299, P13=-5.359680988923596, P14=-1.1139277044637974, P15=6.798910934918159, P16=3.6243910798873933, P17=13.522481024419736, P18=15.897383461913234, P19=20.301140604675663, P20=16.79524793824116, P21=18.857592332559484, P22=11.90124579173308, P23=4.858737944233444, P24=3.618224420207689, P25=0.875802974900621, P26=-3.435272532338163, P27=2.417461029193052, P28=-2.351812413663401, P29=3.6339231753818777, P30=-0.2720607693593516, P31=-0.18288778340691567, P32=-3.2466407259276755, P33=0.5995556637303142, P34=-2.38511529067768, P35=2.2028749121174345, P36=-4.167415763004252, P37=-2.1423595489334173, P38=-4.989732735399151, P39=1.0558185266744882, P40=0.5022977510635538, P41=0.4608919207121205, P42=0.48977788026417973, P43=0.4633030514194696, P44=0.4701627688497001, P45=0.5102851353191845, P46=0.6061543982536346, P47=0.6756868409114006, P48=0.6319083160350204, P49=0.5779088390168093, P50=0.6202388648775665, P51=0.5945715887061466, P52=4.160323289253013e-05, P53=9.972589464850282e-06, P54=2.097286832026647e-06, P55=1.396232291775999e-07, P56=7.993268485244526e-08, P57=3.5217387162072804e-08, P58=1.933896956922607e-08, P59=2.841286466862602e-08, P60=2.2612138949500984e-07, P61=5.610551028256455e-06, P62=0.0002830030239949726, P63=0.001515545890568858, P64=0.0027246139476736625, P65=0.0038136628068042478, P66=0.005515561889523939, P67=0.004248916104138881, P68=0.0015800384963868264, P69=0.00020338237763840855, P70=5.4046350088859985e-05, P71=1.6750533799538416e-05, P72=6.1550013641850325e-06, P73=5.394607823020854e-06, P74=2.9078739323472443e-05, P75=0.00016005413092897805, P76=0.00044995733831080235, P77=0.0013033884106635916, P78=0.002053958311939507, P79=0.0015533158952564798, P80=0.002152834745968235, P81=0.0018717283676932307, P82=0.002098954628394957, P83=0.0012935145708526188, P84=0.0009663685330262362, P85=0.0015383980301126564, P86=0.0002684917116638088, P87=0.0001497896076079243, P88=0.000543925387087145, P89=0.0004133472104276746, P90=0.0003720697483387036, P91=0.00023273476775834983, P92=0.0004551815304767733, P93=0.0007308742818577752, P94=0.0016354368152809748, P95=0.002553306843466638, P96=0.00046031935405754196, P97=0.00038265180060259916, P98=6.551385878035004e-05, P99=2.9829640920561603e-05, P100=0.00010333501992035673, P101=0.0012386103472715826, P102=0.0025931527609634334, P103=0.0016593643372081325, P104=0.00033458116242375346, P105=3.0348658536660605e-05, P106=9.49102821751292e-06, P107=1.4045560986066828e-05, P108=0.0001149595770223629, P109=0.00026178612304964287, P110=8.058822994397324e-05, P111=3.6428692266382426e-05, P112=6.648142878322323e-06, P113=7.318657641360158e-05, P114=0.00014408489269191263, P115=0.00020498108129179949, P116=0.00011605872677193793, P117=5.342878475770874e-05, P118=0.00010783915303478277, P119=2.7378530819529643e-05, P120=1.5012423200878875e-05, P121=5.0966029402330125e-05, P122=4.777054420359242e-05, P123=1.7425806495286373e-05, P124=9.588679463162633e-06, P125=2.4615092478348984e-05, P126=4.548292395752603e-06, P127=3.3320216501386745e-06, P128=5.417565656439755e-06, P129=2.6650050736479065e-06, P130=1.3954554545935418e-06, P131=1.469057956362834e-06, P132=9.64364413543625e-07, P133=1.4134074604934634e-06, P134=2.426551839738038e-06, P135=2.998242704048058e-06, P136=1.938007425172986e-06, P137=4.959084625480731e-06, P138=2.8112544652634518e-06, P139=5.14907787688788e-06, P140=2.657946908069048e-06, P141=1.2594199527945355e-06, P142=7.306688777128536e-07, P143=2.1226746292137286e-06, P144=3.224936342011127e-06, P145=2.830874219294558e-06, P146=2.4208094643988723e-06, P147=1.5853972127130427e-06, P148=1.9322460669905817e-06, P149=1.2214734306912285e-06, P150=7.895915807526514e-07, P151=1.0414009525801815e-06, P152=1.0232520929075393e-06, P153=1.257528026819066e-06, P154=2.3070134598853814e-06, P155=5.3931339238873335e-06, P156=1.2185751178375112e-05, P157=1.2288708580329343e-05, P158=1.144401903713458e-05, P159=1.056967296418831e-05, P160=9.557473874486877e-06, P161=8.051171219836618e-06, P162=8.326610677012625e-06, P163=7.060540393392377e-06, P164=4.721313444842899e-06, P165=9.292144860854086e-06, P166=8.287943578254512e-06, P167=7.309471924090897e-06, P168=9.38882625960632e-06, P169=8.076775059369686e-06, P170=1.0036176350918149e-05, P171=1.2249639519029937e-05, P172=8.214093749748971e-06, P173=3.0185988603533037e-06, P174=4.641580154826988e-06, P175=8.75739526923483e-06, P176=9.437373411724382e-06, P177=8.802940942925575e-06, P178=4.3608827922619435e-06, P179=7.843200245472261e-07),Row( P0=-734.1165342143025, P1=47.644396787430836, P2=-18.85766756008771, P3=-3.8828950559932975, P4=-17.160107435094748, P5=-14.342154741994456, P6=-2.7068090904038167, P7=-0.9142104763005078, P8=-9.05255357252869, P9=-11.906049614340915, P10=-3.9423936147264214, P11=-3.6185788055098858, P12=-7.377383253964971, P13=-4.7385776985198405, P14=0.1687563944859204, P15=5.044025773119211, P16=3.6705034701382506, P17=11.524185002165709, P18=12.44222737955977, P19=16.247853197532034, P20=11.06003416892191, P21=12.624744021415395, P22=9.483442880822677, P23=4.83882124000143, P24=4.475713251911204, P25=-2.13173696205191, P26=-1.2048855065858295, P27=1.117121669756816, P28=-1.0552638476892766, P29=4.2159296365799435, P30=-0.8191916031435008, P31=1.55885193790876, P32=-0.24890259349692648, P33=1.7434514659779365, P34=-2.9312752011442695, P35=-0.4632628194579735, P36=-5.777771036961167, P37=0.8331525335326433, P38=-1.865737800052422, P39=2.9564933758919585, P40=0.6527309756915064, P41=0.627271114782365, P42=0.5299350260247516, P43=0.4823779589607147, P44=0.5283981707659702, P45=0.5693631051106665, P46=0.628845505256638, P47=0.6753898863732104, P48=0.6631449257997957, P49=0.6388898740763596, P50=0.6152643749132248, P51=0.6138955543706998, P52=1.408498062789004e-05, P53=7.419587084472353e-06, P54=7.965455315190216e-07, P55=1.698842398985588e-07, P56=3.4708380205820263e-07, P57=4.704784568616529e-07, P58=1.8456088101239704e-07, P59=1.0013055884066228e-07, P60=8.25162469446383e-07, P61=1.3437010570044015e-05, P62=0.0006779345513964118, P63=0.0018590830882506922, P64=0.0029619311685792436, P65=0.0037620279817538603, P66=0.001860067277125844, P67=0.004343079787567057, P68=0.004671192945545255, P69=0.000659607554106432, P70=0.00013061632428388497, P71=2.3033646160730653e-05, P72=2.973242948392793e-06, P73=6.320752332807077e-06, P74=6.179880249609847e-05, P75=0.00011985193704886827, P76=0.00021003087457646666, P77=0.0004577394758718634, P78=0.0010345141685696151, P79=0.0009394190849979415, P80=0.00031492034520617365, P81=8.721997723260719e-05, P82=0.00025179936989521773, P83=0.0038948626635024615, P84=0.01242802586075311, P85=0.007162889393838325, P86=0.0011642973240394484, P87=0.00028816496957523606, P88=0.00024268092096255676, P89=0.00013610496712019955, P90=0.00011072921981808023, P91=0.0002827573944490737, P92=0.00022370673105509868, P93=6.089694824872974e-05, P94=2.7224407788686542e-05, P95=6.447666055944186e-05, P96=0.00035875261534312286, P97=0.00026250600998207357, P98=7.587130431244019e-05, P99=5.7190730070262294e-05, P100=0.00016704342182691654, P101=0.0001234732038713254, P102=5.274925807178293e-05, P103=2.4287574762674493e-05, P104=1.9532212313291834e-05, P105=4.484262281473655e-05, P106=9.954790357215402e-05, P107=0.0002530885081968221, P108=8.327249045954393e-05, P109=2.2170290737123788e-05, P110=4.666102258207857e-06, P111=8.109306285979264e-06, P112=7.968846822090004e-05, P113=0.00011804171526806398, P114=6.281744553914272e-06, P115=1.0014572862250988e-05, P116=9.442631477088419e-06, P117=2.205971214035464e-05, P118=3.279160730643466e-05, P119=1.4323232617175356e-05, P120=9.498514527509335e-06, P121=2.6062569383918387e-05, P122=2.5449206351996498e-05, P123=1.8610214251567706e-05, P124=1.4120761861511034e-05, P125=1.932492821221462e-05, P126=1.076896602634363e-05, P127=8.427977315536319e-06, P128=6.851199436119808e-06, P129=9.257187007782923e-06, P130=7.771391358244288e-06, P131=6.921654008610666e-06, P132=6.735767056289838e-06, P133=9.244121844506444e-06, P134=1.1817367273902448e-05, P135=1.5226056469570532e-05, P136=1.3327290034057663e-05, P137=9.926060739958108e-06, P138=6.996435900022177e-06, P139=1.3248466704097715e-05, P140=8.68808268054335e-06, P141=1.0219046031293265e-05, P142=6.7636325021032485e-06, P143=3.953517768491932e-06, P144=2.7437623073372585e-06, P145=3.398078108073598e-06, P146=3.420853939747361e-06, P147=5.430614859542407e-06, P148=1.2621093071111227e-05, P149=1.1339287240661206e-05, P150=1.0845952513138524e-05, P151=5.79322738567914e-06, P152=3.394287831114651e-06, P153=2.176493633742272e-06, P154=5.868930206321476e-06, P155=1.675498274118637e-05, P156=2.607811251152779e-05, P157=2.8544702666317926e-05, P158=1.6886057440453805e-05, P159=1.4555954917919796e-05, P160=1.682696009535554e-05, P161=1.0142369851033065e-05, P162=7.250165874592158e-06, P163=6.880611937511819e-06, P164=2.639499130794756e-06, P165=2.6715492428392524e-06, P166=6.753380622059919e-06, P167=1.0088946020825405e-05, P168=1.76182548970685e-05, P169=1.89656052776925e-05, P170=2.37086261195422e-05, P171=2.2062917306247305e-05, P172=3.3276472200172495e-05, P173=7.640316033733203e-06, P174=9.735872079874973e-06, P175=3.0181664085413414e-05, P176=8.82456377293259e-05, P177=2.7787152459799315e-05, P178=7.197615339516435e-06, P179=7.485819822300589e-07),Row( P0=-681.2211120653786, P1=38.191386590434576, P2=-34.89346222184849, P3=-11.617323544933186, P4=-23.258257563400544, P5=-22.921789997300138, P6=-4.803262848648237, P7=-8.147207924152575, P8=-8.486280949818614, P9=-16.643908478156693, P10=-7.727946282624282, P11=5.948344408314735, P12=2.4383087092225812, P13=18.541805211611642, P14=20.50147231981361, P15=19.987955208257624, P16=9.66292622561194, P17=2.9900269627548655, P18=3.370306060596296, P19=4.809874547660853, P20=4.325259949788213, P21=8.023479052190497, P22=0.06445162216471108, P23=-3.449065686866843, P24=2.751029061436675, P25=4.650939425018493, P26=6.282015599595146, P27=5.440891314093141, P28=-1.7760032010759745, P29=0.7577123784852151, P30=0.1274715499184977, P31=2.3161214390048404, P32=-1.859805291337739, P33=0.08423276461052, P34=-2.398495936277303, P35=1.9963237141131847, P36=-1.3725159795619966, P37=2.1334166506320513, P38=-2.2956497095765123, P39=2.089017005219788, P40=0.6318684881353921, P41=0.646982523690856, P42=0.6392752973407744, P43=0.6516056568230376, P44=0.6597698120643906, P45=0.5940114820403467, P46=0.48286376831939737, P47=0.44297692034301994, P48=0.4597586900944472, P49=0.5211112936810507, P50=0.6395594261191827, P51=0.6409738568350055, P52=7.086170269814292e-06, P53=2.4928136068233136e-06, P54=7.118622115607488e-07, P55=2.456823251235152e-07, P56=2.263147520463685e-07, P57=2.7975821274207814e-07, P58=1.1639632592681867e-07, P59=3.306122691639432e-08, P60=1.0620669222211361e-07, P61=1.3071473379205584e-06, P62=6.269426400806306e-05, P63=0.00031565992708841286, P64=0.0004728445506742418, P65=0.002223617772677497, P66=0.006965313673929097, P67=0.0124112889087504, P68=0.013027515068271218, P69=0.023064314141901944, P70=0.03425122086195862, P71=0.03375441253995456, P72=0.02954059999948105, P73=0.015249764145645524, P74=0.0033628677640785004, P75=0.0008294422007071553, P76=0.00021346126852178355, P77=6.73565904531268e-05, P78=8.352175478245418e-05, P79=0.0008579850923686208, P80=0.0031801684739572113, P81=0.0030653231291807017, P82=0.003269517508265659, P83=0.0028060751962802727, P84=0.0029702774818458092, P85=0.007951346809203477, P86=0.008298369986530184, P87=0.008845692554208619, P88=0.01633434768494452, P89=0.021960633957835766, P90=0.018643495634424953, P91=0.02091210613446853, P92=0.027182843893768956, P93=0.016042963635364862, P94=0.003357957926789437, P95=0.0016027558704275258, P96=0.0010764294313205737, P97=0.000995934716252402, P98=0.0073978558358581145, P99=0.009047997297446929, P100=0.0058923748542137775, P101=0.018945995649450396, P102=0.02428746811642848, P103=0.0045706450304609765, P104=0.0012418389832163383, P105=0.000517735161596898, P106=0.0010485288500029248, P107=0.007331639561634759, P108=0.001765800808127864, P109=0.0004327028481526601, P110=0.0010791491512353694, P111=0.0007939204504301503, P112=0.0004348375253414495, P113=0.0003419370266232801, P114=0.000703457293885428, P115=0.0005692129359067881, P116=0.0007618000855925342, P117=0.0010948101064851329, P118=0.0007690395493694243, P119=0.0011388728515073284, P120=0.0015712078071247507, P121=0.0005918803039651784, P122=0.0018081902592567407, P123=0.0010597206431651042, P124=0.0012086009726443117, P125=0.0021190273951384514, P126=0.0010831619636672237, P127=0.000299455384022625, P128=0.00028307367538399906, P129=0.00023773618847683543, P130=8.972647417905004e-05, P131=7.723660811456182e-05, P132=0.00028546855387782917, P133=7.856218116630331e-05, P134=0.00010113083112770178, P135=0.00014546103465846015, P136=0.0001069444511103743, P137=0.00022516492701017706, P138=0.0001443006695162546, P139=0.00015532778162214873, P140=3.469870578630306e-05, P141=5.202650675404452e-05, P142=3.562164888701609e-05, P143=2.7215222709416992e-05, P144=2.537413011142749e-05, P145=4.061387532578597e-05, P146=5.4876727874269634e-05, P147=6.941880582711198e-05, P148=9.3815167715695e-05, P149=7.970009054031183e-05, P150=5.121170264890884e-05, P151=1.866512295435407e-05, P152=1.2605444574246576e-05, P153=4.452082792654896e-06, P154=2.787469119299285e-06, P155=2.854698117649732e-06, P156=3.7599186243766523e-06, P157=6.954458468246145e-06, P158=9.561237150367957e-06, P159=1.1188679796014948e-05, P160=2.581614842157218e-05, P161=2.9280820080150625e-05, P162=1.6478460809267772e-05, P163=1.28676430796853e-05, P164=1.0261586763441209e-05, P165=7.809061668462186e-06, P166=1.0564011655753418e-05, P167=1.2496813755383289e-05, P168=1.5717017880780344e-05, P169=1.3514722169793998e-05, P170=1.7638650471168278e-05, P171=2.327704298012256e-05, P172=2.976885078640417e-05, P173=2.8114141759641934e-05, P174=2.3537626423532155e-05, P175=7.306214650196591e-05, P176=0.00028783203639000084, P177=0.00025068301321695414, P178=0.0001436654265113371, P179=9.601279508033182e-06),Row( P0=-626.9968198600619, P1=37.528151775841714, P2=-30.76794025484581, P3=-9.476089257587423, P4=-18.97206887435153, P5=-16.34023159496003, P6=-4.4153071288593635, P7=-5.293879841527318, P8=-13.932440513974802, P9=-8.707229021026441, P10=-3.776713527754861, P11=3.457615308895015, P12=1.4899127089695785, P13=12.958235606352488, P14=5.8861134582225, P15=8.007482394452412, P16=5.1011887220489145, P17=2.3978460236140955, P18=7.706967215344903, P19=3.611894965995898, P20=7.773960688625593, P21=6.338368276307821, P22=1.968801333010198, P23=2.2995998441069565, P24=2.9305101270167797, P25=2.423861059571877, P26=0.837553192279496, P27=3.0602950318493525, P28=0.8579067495010333, P29=1.7428135645215574, P30=0.9518281875546182, P31=2.9409067154939907, P32=0.3867377941614286, P33=1.8803875961243564, P34=-2.9240375317345837, P35=0.5337033552031131, P36=-1.8020694396499461, P37=0.3065716987577374, P38=-2.5866112218955513, P39=-0.9292321508276563, P40=0.6417736424899447, P41=0.6655754091352154, P42=0.6622484082690948, P43=0.6673868558909763, P44=0.709742100866247, P45=0.6644411033195666, P46=0.6113548927767529, P47=0.5829451483251272, P48=0.5711154064871289, P49=0.580575337697134, P50=0.6106774758397708, P51=0.629311482212653, P52=1.387916557099135e-05, P53=1.8862636365620924e-05, P54=5.959604457728575e-06, P55=6.811363930952374e-07, P56=1.5867467292923778e-06, P57=4.533052122906516e-06, P58=3.0584676166283857e-06, P59=1.8466683977070492e-06, P60=6.574593938538e-06, P61=3.259878202513454e-05, P62=0.0003182717102985885, P63=0.0008304306810371206, P64=0.0005803632397710671, P65=0.0013719417795277973, P66=0.005871774801715675, P67=0.012799594841675862, P68=0.010633203795843845, P69=0.01916891547533806, P70=0.027380384346274326, P71=0.021526410738332172, P72=0.017459840019883436, P73=0.0158744916008939, P74=0.061592416357553297, P75=0.09005379370367476, P76=0.04038439786001066, P77=0.03768411004891243, P78=0.015500556087543466, P79=0.003763615569910649, P80=0.004558427767898035, P81=0.009006018028915855, P82=0.012460808289426185, P83=0.013593847323454894, P84=0.009264517004790536, P85=0.009510096906803375, P86=0.01678966715096954, P87=0.017997079179569296, P88=0.016005812355449064, P89=0.021829340918844045, P90=0.02309507346158304, P91=0.024176935879797606, P92=0.03311910910719277, P93=0.028999813844127153, P94=0.008955169460736398, P95=0.008200558801006593, P96=0.008451338271176126, P97=0.011264257761802203, P98=0.031874185590822494, P99=0.05806622367144692, P100=0.029409992868330286, P101=0.0601857539428427, P102=0.04502090526249852, P103=0.01353507772050511, P104=0.00515129138288016, P105=0.0028814131845928876, P106=0.022967180361107984, P107=0.03785353430091961, P108=0.015008071908719443, P109=0.010105216295199313, P110=0.002122443342370547, P111=0.0008157923202532754, P112=0.0017345655877843262, P113=0.010727235836319874, P114=0.013580151022830724, P115=0.003465272571710396, P116=0.0037926190266570083, P117=0.0018869516430168206, P118=0.0006339408495825369, P119=0.0031207095365396315, P120=0.00101868991186049, P121=0.00031463741128583947, P122=0.0002327691933197944, P123=0.00039534738590936696, P124=0.0013552003569572149, P125=0.004127067911010985, P126=0.0015965188868262467, P127=0.0004511348559285626, P128=0.0006516913115989681, P129=0.00289502685238209, P130=0.0010588795847203882, P131=0.0007870258202501904, P132=0.0011951653831677566, P133=0.0017471938362332473, P134=0.0015171648730589635, P135=0.001682656119853058, P136=0.0011491811010425686, P137=0.0006249185586165646, P138=0.0003746585360212021, P139=0.0002576366096648321, P140=0.00022140343266701504, P141=0.00011685139585526398, P142=0.00014518356498933598, P143=0.00011192652383125665, P144=0.00011908749985030494, P145=0.00016261012019114452, P146=0.00014251080459515807, P147=0.0001636078377278175, P148=0.00021327212152684984, P149=0.0002543590957649393, P150=0.00036376624717298, P151=0.0002812250718566462, P152=0.000190099991841775, P153=7.792284775616792e-05, P154=7.888488325087592e-05, P155=9.505066572964846e-05, P156=4.339863897114184e-05, P157=3.020408741093447e-05, P158=2.5221493697569172e-05, P159=3.334889162618707e-05, P160=6.908485005657592e-05, P161=8.002417934876937e-05, P162=7.947703391839847e-05, P163=0.00014238555753976598, P164=0.00012112735904683314, P165=0.00010586496759684966, P166=0.00011160059597508253, P167=0.00013466374836936402, P168=0.00014173551217115305, P169=0.00011489792333473133, P170=0.00011967974581758291, P171=0.00012499042882925074, P172=9.587721964314443e-05, P173=8.008438187040627e-05, P174=0.00017109068644668998, P175=0.00025827308725506315, P176=0.0002234348624835237, P177=0.00017968058808513814, P178=8.412146370383068e-05, P179=9.087061522194751e-06),Row( P0=-723.4221638086024, P1=59.58335375693774, P2=-21.794766528546976, P3=-6.881589671581575, P4=-15.131082678848303, P5=-17.329954613329015, P6=-4.861573534491762, P7=4.799280678311206, P8=-7.7661390197485565, P9=-10.689058825826404, P10=-3.714019831658774, P11=4.142444730858728, P12=-7.361659468503664, P13=-4.20991269328631, P14=-1.2783945735938038, P15=4.401372807897394, P16=7.784726491336232, P17=16.54339090468827, P18=16.209067121285567, P19=22.397403211999276, P20=18.235883709725062, P21=12.89105994047733, P22=6.277199544201935, P23=0.9723812539782702, P24=3.052347599516092, P25=0.45893804016280165, P26=-1.5710041232155236, P27=1.625280053991573, P28=-2.859347743489366, P29=1.109728274887623, P30=-4.376312315062026, P31=-1.2251518902809237, P32=-4.274548591799534, P33=-0.09337656314194608, P34=-3.221300272737995, P35=0.1362817523305453, P36=-6.206890296769438, P37=-0.9500188431705264, P38=-4.617188769540335, P39=2.166402604451129, P40=0.6133717442925142, P41=0.4941345801609717, P42=0.46108495763826174, P43=0.5015312880806112, P44=0.47288258948068695, P45=0.4749003279564997, P46=0.5177091122437361, P47=0.5725416738708846, P48=0.6928955588762525, P49=0.649508293279105, P50=0.6731281124398997, P51=0.6652457526548333, P52=4.5034949320531686e-05, P53=5.5723815090734496e-06, P54=1.6973836028724893e-06, P55=1.041128825394264e-06, P56=6.741510377948176e-07, P57=1.7701643667666479e-07, P58=3.094462452856476e-08, P59=4.6755165122740064e-08, P60=4.665533156347613e-07, P61=4.452871872549344e-06, P62=0.00020548308563256762, P63=0.0012990998717699804, P64=0.002942274258691357, P65=0.006475245342598386, P66=0.008811696409906167, P67=0.0075401149101359905, P68=0.0016535072982983077, P69=8.445626831448515e-05, P70=3.647627652664208e-05, P71=1.333788836693511e-05, P72=7.394614710766196e-06, P73=1.1200662358742028e-05, P74=5.352342424761005e-05, P75=0.0001623760204975101, P76=0.0008080883068970052, P77=0.0030895176770439205, P78=0.002601802271559922, P79=0.003428178483385008, P80=0.004164251300419048, P81=0.004373819089706131, P82=0.004648043143861089, P83=0.0026327554108128053, P84=0.0007514263892337757, P85=0.00031284857052591764, P86=9.930952794652707e-05, P87=0.00012066764484078524, P88=0.0010103533589342886, P89=0.0029874200520170176, P90=0.001779745167020252, P91=0.00039681142613928944, P92=0.00045205174276148483, P93=0.0013351834564432911, P94=0.0023011495219929087, P95=0.0011614924627617615, P96=0.00013915471714859686, P97=0.0008058085040981788, P98=0.0018149878342075669, P99=0.00024047822775004403, P100=2.697301414336993e-05, P101=0.000587466224261653, P102=0.0028371868408170737, P103=0.0011356186968240988, P104=0.00020229408645278276, P105=0.0002244518900953575, P106=3.147420164330401e-05, P107=6.5677260139623565e-06, P108=8.955178102893739e-05, P109=0.00018036723043383823, P110=6.966026036421087e-05, P111=2.345960580701363e-05, P112=3.7825721012099358e-06, P113=7.115026224830987e-06, P114=0.00010197919549837917, P115=0.00023928993074113044, P116=8.692069469043232e-05, P117=8.043618811786112e-06, P118=3.661337229209113e-05, P119=0.00023310970740714077, P120=6.346806821797868e-05, P121=1.6659685609712105e-05, P122=4.6569594220142575e-05, P123=8.338695086786613e-05, P124=1.796779394793355e-05, P125=2.946680286408459e-05, P126=8.122725381326187e-05, P127=1.503240756161638e-05, P128=1.8669132951571333e-05, P129=5.5842973984378886e-05, P130=5.42691559939128e-06, P131=6.857597078874026e-06, P132=7.447511861534932e-06, P133=5.587998126502225e-06, P134=1.6737001828769902e-05, P135=7.178845815414343e-06, P136=6.127939406175058e-06, P137=7.387545925144104e-06, P138=3.5204284604643847e-06, P139=3.881970167062251e-06, P140=1.9645082779040563e-06, P141=2.976044023205867e-06, P142=4.039890734642506e-06, P143=1.92969820035849e-05, P144=1.635580423070441e-05, P145=7.485140134723501e-06, P146=3.4234582220277105e-06, P147=3.4880276986023783e-06, P148=4.362460965601557e-06, P149=3.128792989711511e-06, P150=2.50616700146677e-06, P151=8.934505759409708e-07, P152=1.7399712288109414e-06, P153=9.479180518270591e-06, P154=1.1184499038060228e-05, P155=9.76678022014999e-06, P156=7.1906339481641475e-06, P157=1.0167185656605899e-05, P158=1.2137294595732106e-05, P159=1.2675704416903756e-05, P160=1.9864229659384525e-05, P161=7.837850632102567e-06, P162=1.0646738860919666e-05, P163=1.4075859972481886e-05, P164=8.419077326061714e-06, P165=9.957664457761305e-06, P166=1.4514304949856757e-05, P167=1.5917708796618536e-05, P168=1.4134060572147706e-05, P169=1.5906574949466403e-05, P170=1.3026043502009605e-05, P171=2.1801307223882134e-05, P172=2.5173147419096188e-05, P173=1.1838691704940547e-05, P174=1.6117190124863595e-05, P175=2.5895859422345767e-05, P176=3.136459723481322e-05, P177=1.951847396889874e-05, P178=6.538277250308768e-06, P179=7.198854825708169e-07),Row( P0=-482.65571882865174, P1=-2.0244602501571056, P2=-58.25052313874825, P3=-5.569172086288088, P4=-23.166778621176547, P5=-18.00790034528345, P6=13.407268542295204, P7=14.136723415726319, P8=10.050764955919743, P9=5.290967828533135, P10=-4.287674245964663, P11=2.1255913743509742, P12=-9.050156803077353, P13=13.802522638263708, P14=15.406530941514415, P15=4.845901418202828, P16=-4.651897243144462, P17=0.16368538691340115, P18=11.612361554429084, P19=8.251214727469431, P20=6.866551583024952, P21=7.935851964368622, P22=1.8831494445723773, P23=1.0420134034041855, P24=2.309025894520342, P25=1.0731994581239857, P26=7.207313320048847, P27=6.665377535963353, P28=0.6500967308526103, P29=2.5782382507937283, P30=3.790417818346501, P31=3.4698716435983883, P32=2.1428361514425274, P33=4.725938787654386, P34=0.03001478889326574, P35=3.1856890088918783, P36=-0.019207951649102613, P37=4.4145208063880625, P38=-0.8098688956607429, P39=2.1681703443547917, P40=0.6497332161933906, P41=0.6206475068501565, P42=0.6007237341712496, P43=0.562545337964496, P44=0.5631018475237362, P45=0.5008271131522475, P46=0.44085060984835095, P47=0.4554826958156131, P48=0.46898963852616316, P49=0.5101752565340391, P50=0.5620321756265051, P51=0.6099476969380814, P52=0.0038107775699478554, P53=0.0003056071083463795, P54=1.1948942373011634e-05, P55=3.0470125903083034e-06, P56=4.146553338256779e-06, P57=2.3803419675032847e-06, P58=9.661664800761787e-07, P59=1.2602995505748125e-06, P60=4.901199233277352e-06, P61=1.2028174133825348e-05, P62=2.1386421448525304e-05, P63=3.5015682855495384e-05, P64=2.578531279024296e-05, P65=3.182520871624803e-05, P66=5.5053669843864176e-05, P67=7.474910356055804e-05, P68=4.348438798057804e-05, P69=0.00010754362017894245, P70=0.0003226427008837842, P71=0.001585951989485587, P72=0.0021130913200043604, P73=0.002335721754929454, P74=0.002643705629548438, P75=0.002395443194689464, P76=0.0023679041895999695, P77=0.005459145654925605, P78=0.010045198882605732, P79=0.012923434392414706, P80=0.01731298142364057, P81=0.022321665899337668, P82=0.03291281854391567, P83=0.03830234312807723, P84=0.08010517518206878, P85=0.2209450812870098, P86=0.33489746449027685, P87=0.5827909426755232, P88=1.020520743095516, P89=2.4049305720447296, P90=2.885550593560714, P91=2.689755588683155, P92=3.892129606574208, P93=2.254169793866374, P94=0.0866617309832835, P95=0.018372418389040697, P96=0.012002712164163216, P97=0.012352731808617375, P98=0.025645628344581988, P99=0.03814003021159485, P100=0.05647939191159526, P101=0.11828576219334246, P102=0.12778072504407403, P103=0.0895459006007407, P104=0.0862647604629094, P105=0.14004370259708113, P106=0.39904426193568215, P107=0.8371146241010319, P108=1.7737661699027636, P109=1.13209032517967, P110=0.1593086004113873, P111=0.04548860068355845, P112=0.08224125949247166, P113=0.14039276954909138, P114=0.09850899074218142, P115=0.02303725311549396, P116=0.017796042601503886, P117=0.3611379086566519, P118=0.7495446676567904, P119=0.367101087943104, P120=0.5566724591318107, P121=0.8780145395168418, P122=0.5041602099557451, P123=0.2640594203386685, P124=0.14637294965432388, P125=0.1428609776407867, P126=0.3874841807844754, P127=0.13054521006049843, P128=0.16594521399723197, P129=0.3866559269730236, P130=0.12427368263307098, P131=0.027980839925276977, P132=0.03708073134866666, P133=0.0321306196559921, P134=0.05496427611360186, P135=0.073006373751356, P136=0.052923359365076535, P137=0.05250267032692048, P138=0.023182342539813234, P139=0.03295158534129582, P140=0.02843688536957621, P141=0.04330978608497157, P142=0.025499873025289242, P143=0.007877078016593238, P144=0.008236057922863447, P145=0.012386498157206778, P146=0.01289239355799514, P147=0.01450435941240123, P148=0.010424742519566469, P149=0.007682908182719923, P150=0.01205158469624394, P151=0.008013089682135937, P152=0.0017352366863142283, P153=0.0006749568826467115, P154=0.001060901274184783, P155=0.00040189271257884906, P156=0.0001796729195783095, P157=0.00013020066402502135, P158=0.00014126981396293575, P159=0.0002187254537509576, P160=0.00019629685838485513, P161=0.00017815722054027183, P162=0.00019733628389066832, P163=0.00018415247863258374, P164=0.00016245213580389167, P165=0.00018451496035387196, P166=0.00015838323317075024, P167=0.0002828311508498293, P168=0.0005579776066545666, P169=0.0007729242057194278, P170=0.0005427143399274725, P171=0.0004889923440631993, P172=0.00040671220224835044, P173=0.0002794437081412221, P174=0.0004296215241696459, P175=0.000840113519986384, P176=0.0022056926993736523, P177=0.002953400248107929, P178=0.0014410168664646785, P179=0.00011083073927833743),Row( P0=-732.7495000220256, P1=50.65686035974234, P2=-19.426927272531138, P3=-15.8669317893644, P4=-19.022143022116644, P5=-21.1014783274093, P6=-10.523467680719438, P7=-3.030891174724891, P8=-11.74178903313924, P9=-16.844260579951545, P10=-7.996751506472202, P11=-3.4917565480960673, P12=-10.649146011571077, P13=-5.1483310993730065, P14=-1.6992204359385226, P15=4.587546147213077, P16=6.982929778333633, P17=13.126916612055695, P18=13.64680052092031, P19=16.544826874118463, P20=13.64246582266763, P21=11.556276055667563, P22=5.032798133501764, P23=2.975173595525187, P24=4.675565387073896, P25=-0.4386521606800259, P26=-1.3981480823756927, P27=0.5306576048800686, P28=-1.1772479659766166, P29=3.271490477303778, P30=-1.12683468426824, P31=0.8248965381821606, P32=0.02475180267911293, P33=2.319059116054979, P34=-2.1549818160008587, P35=-0.45816450486243165, P36=-6.377492270336984, P37=0.7895150123112215, P38=-3.1250161199806623, P39=2.5246380398648607, P40=0.6642624383648432, P41=0.5614082812800695, P42=0.48099890466236483, P43=0.44841180453852897, P44=0.48042500855640224, P45=0.4863284188464171, P46=0.5268497202384056, P47=0.5895108990553745, P48=0.6019240698641657, P49=0.6245078867506468, P50=0.6250431637452403, P51=0.6212087375332325, P52=6.0972579836592565e-06, P53=2.3753023092784724e-06, P54=9.850144182546396e-07, P55=6.334260952791144e-08, P56=7.1649393216924e-08, P57=1.1035498799963401e-07, P58=6.933219723221203e-08, P59=5.7751065351264665e-08, P60=4.546422914449237e-07, P61=1.1124144390496657e-05, P62=0.0005282940844200303, P63=0.0015272006831219382, P64=0.002476242004710945, P65=0.006238363485132274, P66=0.007929157449943243, P67=0.02167175990137134, P68=0.010695733376859059, P69=0.0007156732644450865, P70=0.00013807951077699946, P71=2.5672663375427755e-05, P72=6.019858811967638e-06, P73=8.72984443731821e-06, P74=0.00014280372301795063, P75=0.00038138791650627654, P76=0.0005245130969192213, P77=0.001745142495362778, P78=0.002247377652267914, P79=0.003764613089232468, P80=0.004607766113431848, P81=0.0039025892124069776, P82=0.0030204243617258323, P83=0.005403314490155147, P84=0.00424727678709776, P85=0.00265177911546832, P86=0.0011557925963024032, P87=0.0003109718076604533, P88=0.00038854558384488613, P89=0.0013677218869093959, P90=0.0009729347428410627, P91=0.0007321339627295773, P92=0.0005223616238470672, P93=0.00011885239596240899, P94=6.942512809833445e-05, P95=0.0043229301355416465, P96=0.008262280608854403, P97=0.0009277115826511081, P98=0.0010701798880683123, P99=0.0003115009587336505, P100=6.308105766371687e-05, P101=4.096186215931213e-05, P102=3.730416277516426e-05, P103=6.262229405098911e-05, P104=0.006111388857423396, P105=0.004182137464868457, P106=6.458378999359704e-05, P107=1.641201771594413e-05, P108=7.070484776664965e-06, P109=6.295528422512906e-06, P110=5.0714598734528715e-05, P111=8.45710757298833e-05, P112=6.4104726820207e-05, P113=1.0638241149364688e-05, P114=8.62109156950792e-06, P115=5.043001637738142e-05, P116=6.874448927634e-05, P117=0.00041988983866754687, P118=7.162545735737626e-05, P119=9.925194856601744e-05, P120=1.2601217705422742e-05, P121=1.996164067696798e-05, P122=6.939912194585377e-05, P123=5.436757175114236e-05, P124=6.8269353006485794e-06, P125=1.0949147614049903e-05, P126=2.7980456779022196e-05, P127=7.692126609815316e-06, P128=4.833436155844623e-06, P129=9.50945648817325e-06, P130=3.353300384161224e-06, P131=1.6199945028398763e-06, P132=2.1496934110473786e-06, P133=2.4072791140438137e-06, P134=3.825462046821761e-06, P135=2.8971853229399477e-06, P136=2.4996164150273207e-06, P137=2.8106697642412384e-06, P138=3.7893845792553446e-06, P139=4.1312666150063556e-06, P140=2.884683179963982e-06, P141=2.460491425717834e-06, P142=1.1556371594829634e-06, P143=1.1946611867968716e-06, P144=1.0139185145966595e-06, P145=2.1163887771478937e-06, P146=3.1766059423310793e-06, P147=4.583520204879328e-06, P148=8.163208286551028e-06, P149=1.4943180417678316e-05, P150=5.7316453257700015e-06, P151=2.7894292603474925e-06, P152=1.9457247319426082e-06, P153=3.200239763975947e-06, P154=5.576360147223573e-06, P155=1.1108909972218355e-05, P156=1.4006254523789053e-05, P157=1.7316028693886027e-05, P158=2.369986445152044e-05, P159=3.777331182088028e-05, P160=4.323126094064963e-05, P161=3.597139245607944e-05, P162=3.1216044776040244e-05, P163=1.567605371098684e-05, P164=8.121078471468602e-06, P165=8.204909523465184e-06, P166=1.9432693648506475e-05, P167=4.2984127221752813e-05, P168=5.476224560630609e-05, P169=6.249201785069339e-05, P170=4.8487119806113806e-05, P171=4.373113750130455e-05, P172=3.793592658525147e-05, P173=2.352479528744124e-05, P174=5.679628594366356e-05, P175=8.581505435427918e-05, P176=9.143325511759349e-05, P177=4.607446186546557e-05, P178=2.0171197279678014e-05, P179=1.7381247247414347e-06),Row( P0=-643.4676916957688, P1=43.91226818015167, P2=-51.597045273343944, P3=6.991524039017076, P4=-29.71887195412239, P5=-23.906873300626536, P6=2.860043453733971, P7=-5.056145767401212, P8=-8.520090099929277, P9=-11.61434595301747, P10=-3.15914720300201, P11=2.2259905119715913, P12=-7.6103968275292635, P13=7.181323791941488, P14=6.87257010042231, P15=7.8275855044485665, P16=10.975843146253787, P17=11.220878140668605, P18=16.583719061766523, P19=12.328039662233312, P20=6.213032464548885, P21=6.124922868990078, P22=-0.24140614570864452, P23=0.4619067673417472, P24=4.1851693660638905, P25=-2.58248037911955, P26=-1.8674184157465543, P27=-0.7424508374072705, P28=-1.117286721069854, P29=3.834300619266617, P30=1.576960385240109, P31=3.3587585289717667, P32=1.8984651368296315, P33=4.41431624160562, P34=2.7465795319274293, P35=3.6815984291709487, P36=-1.0287171642755069, P37=-0.7746621635363209, P38=-3.6402169731106495, P39=2.2609221549205505, P40=0.6701631553044396, P41=0.5885886304756225, P42=0.5625018887899091, P43=0.4910877527743675, P44=0.5135667563638364, P45=0.49240366320071743, P46=0.5424795434083026, P47=0.5728937947389757, P48=0.5088783342881037, P49=0.4813963921254058, P50=0.5897219431277702, P51=0.694367240649114, P52=3.297625996864348e-05, P53=8.935079178354115e-06, P54=2.5682218880097502e-06, P55=1.8115734781714306e-07, P56=3.244811892830533e-07, P57=6.95757076962367e-07, P58=3.389781017283878e-07, P59=1.161130773489732e-07, P60=7.87436640602989e-07, P61=3.6678788103050946e-06, P62=0.00011498745686233338, P63=0.0004156407730090967, P64=0.0004313460306696748, P65=0.0027236273307779508, P66=0.011838950817703638, P67=0.022077357368032395, P68=0.011135379449042951, P69=0.005466769457721521, P70=0.004585344697103734, P71=0.00447875706728655, P72=0.0048346138154420635, P73=0.0034844486431825014, P74=0.01337265283908413, P75=0.041659311121622494, P76=0.04711313703889777, P77=0.009261198130110395, P78=0.0019400440938082556, P79=0.003914005230407103, P80=0.01049320305481144, P81=0.015538072678097871, P82=0.012648635698112648, P83=0.007596248406719124, P84=0.002990958647990725, P85=0.003943853067508523, P86=0.0017133059239022217, P87=0.001121968016436828, P88=0.0019796112836791757, P89=0.004011272292551323, P90=0.01363921782566703, P91=0.028561086812628523, P92=0.009454072411987861, P93=0.006020266265092347, P94=0.012374602306031652, P95=0.05286271489900619, P96=0.03212335507342211, P97=0.02545221337241764, P98=0.0074041620299370664, P99=0.0018910149800779557, P100=0.0005842346877398549, P101=0.0007556717384771925, P102=0.0009699489228619267, P103=0.001747506278343362, P104=0.024221305064619298, P105=0.010875407372450073, P106=0.017647331663284204, P107=0.007770335334240519, P108=0.0009606877851537104, P109=0.0005812230301019126, P110=0.00035925557106075186, P111=0.0007130768330707223, P112=0.005577574824413691, P113=0.011598135263500111, P114=0.0010597591661742527, P115=0.00012543783691261145, P116=0.0010211393424571344, P117=0.0016838003067653079, P118=0.006555909849761897, P119=0.006816685589641052, P120=0.0010193689076841027, P121=0.009533915051367404, P122=0.005185331376491413, P123=0.001317139806284141, P124=0.0009761594202540216, P125=0.0027692562100801515, P126=0.0029703624612021245, P127=0.0003913935995491592, P128=0.000556667769075224, P129=0.0018843291718939872, P130=0.0016082069253755158, P131=0.0003150977661449136, P132=0.0005386208327151582, P133=0.0004223188276702112, P134=0.0006379069177565825, P135=0.0014368780688242715, P136=0.0003167945071701525, P137=0.0006673086404114591, P138=0.0005769830369608195, P139=0.00035976828565876666, P140=0.0003501069701704933, P141=5.6229439783442356e-05, P142=9.416219235938493e-05, P143=7.699691554817863e-05, P144=8.760507335131619e-05, P145=0.00012178108044842629, P146=0.000283548908977136, P147=0.0001883433066184441, P148=0.00045884782335216495, P149=0.00024531142500467014, P150=0.00022564073069311063, P151=0.00010752070464842854, P152=4.362387123232288e-05, P153=1.6108921640786637e-05, P154=9.457632967984619e-06, P155=8.118224154878482e-06, P156=6.19953877587485e-06, P157=5.846348542863986e-06, P158=9.910091242419678e-06, P159=1.6974877855447318e-05, P160=2.4268850777731484e-05, P161=1.3630883947597942e-05, P162=6.517445899047563e-06, P163=8.717100682958081e-06, P164=7.867699288285041e-06, P165=3.385425370253309e-06, P166=2.604153161493623e-06, P167=1.87288732765975e-06, P168=2.2167497542782e-06, P169=3.661623443376429e-06, P170=4.791314410810578e-06, P171=6.526088298883519e-06, P172=4.073645461142611e-06, P173=2.8831519877814724e-06, P174=1.2765755621157192e-06, P175=2.8380501924591043e-06, P176=5.893388529124697e-06, P177=5.0111560235035855e-06, P178=2.393143601923108e-06, P179=2.792832131357445e-07),Row( P0=-706.0656967716352, P1=39.71643012680443, P2=-22.873770961794676, P3=-2.3610372685816925, P4=-12.149297732525996, P5=-14.788694067718902, P6=-3.3382267550375353, P7=-5.060111686667532, P8=-8.073525819470385, P9=-11.968488662198583, P10=-5.85219691410184, P11=3.009465962781077, P12=1.220345520341966, P13=12.343960227021446, P14=14.282085254013616, P15=15.196472946009042, P16=8.809667597302777, P17=7.240694327113068, P18=6.166290758549187, P19=8.395227719619552, P20=4.940335273123781, P21=4.5629877822552976, P22=-2.3276049642494203, P23=-1.8372814137863454, P24=1.7462482681153175, P25=2.8256174895202113, P26=4.257023879950008, P27=3.351801728189679, P28=-1.0957342983738678, P29=1.5817178269227672, P30=-0.12343767230556917, P31=2.215502032977814, P32=-1.6036077991803859, P33=1.6977220318676745, P34=-0.8645249853218373, P35=1.0403898840315808, P36=-2.7952884622909555, P37=1.7556763409610663, P38=-1.6145228062209978, P39=1.0832749892795184, P40=0.7097374067756904, P41=0.7051739788273321, P42=0.6776851814587473, P43=0.6607161798361679, P44=0.7010992062298728, P45=0.6144638568906214, P46=0.5526913777742928, P47=0.5340127658096588, P48=0.5427429276029045, P49=0.5945212951959543, P50=0.6287482566488005, P51=0.686999837817281, P52=3.158568815604833e-05, P53=1.676736705268224e-05, P54=3.123068223428193e-06, P55=8.485137275055266e-07, P56=1.000042627637327e-06, P57=5.126826903702323e-07, P58=3.277918913039207e-07, P59=4.931936484216163e-07, P60=3.358027152584746e-06, P61=1.0914029517567919e-05, P62=5.7774061820428265e-05, P63=0.00015103108672935028, P64=0.00036777565379516827, P65=0.002270185545335107, P66=0.009007793756439158, P67=0.015934224632889626, P68=0.010181168767683529, P69=0.013314944320590475, P70=0.014355549614551832, P71=0.027174422584417334, P72=0.025835937978381827, P73=0.00883070825449112, P74=0.0011442981227385916, P75=7.864978375228798e-05, P76=4.86226907720164e-05, P77=0.00011198698948019957, P78=0.0006553176218561608, P79=0.00115451400928977, P80=0.0010536438462654743, P81=0.001123666365662922, P82=0.0016918176755421413, P83=0.001851394989552901, P84=0.0016219240101968783, P85=0.001512940058940415, P86=0.001481913577498663, P87=0.00227764539005757, P88=0.003653997103819952, P89=0.007631774248456402, P90=0.013231277899241368, P91=0.01554578665341131, P92=0.007983955399019447, P93=0.0024630521220905207, P94=0.000557451155800315, P95=0.001112320715793461, P96=0.0014195279273991914, P97=0.0006700646934784685, P98=0.0013333186219388693, P99=0.004775593369225703, P100=0.006530468710049241, P101=0.003258299983847631, P102=0.0006875501079797417, P103=0.0005233149126133677, P104=0.0005756272261652352, P105=0.00047737805522008126, P106=0.0009517940346046321, P107=0.002014840070301274, P108=0.002676842831112058, P109=0.0017761347229279748, P110=0.0001536683238611209, P111=0.00023108149411842286, P112=0.000411562184728809, P113=0.00030108306190569466, P114=0.00039167709440799173, P115=0.0018958246235069565, P116=0.0012922704306848423, P117=0.0006055525236755572, P118=0.0003744659868071503, P119=0.00044771362821739393, P120=0.0002313625331042555, P121=0.0004417271154303639, P122=0.00025520980940115304, P123=0.00012722755382174684, P124=0.0003328929116030751, P125=0.0005536988006171185, P126=0.00026109548827563757, P127=6.421194370329796e-05, P128=0.00018709082644658068, P129=0.00023738231583850467, P130=0.00011466623688166556, P131=5.192009456616219e-05, P132=0.00011439432659408999, P133=0.00019378723542103914, P134=0.00018062630070510787, P135=8.126477193179851e-05, P136=5.9195051489136093e-05, P137=8.97034056473021e-05, P138=5.383608582636856e-05, P139=6.571045567850969e-05, P140=5.5467749660477795e-05, P141=2.517729448035513e-05, P142=2.367314855869937e-05, P143=2.1549147043226807e-05, P144=2.0305392673187667e-05, P145=2.548513938898088e-05, P146=2.0413932649364567e-05, P147=2.4112957218887117e-05, P148=2.3503229093994516e-05, P149=3.82545073430199e-05, P150=2.672237234936695e-05, P151=1.8986820689100883e-05, P152=1.8271643825454214e-05, P153=8.660287355781893e-06, P154=6.4513998838226065e-06, P155=5.243012446320777e-06, P156=6.583189833953569e-06, P157=7.871445037367884e-06, P158=1.4376035039431393e-05, P159=2.3781436595903678e-05, P160=2.454788799770117e-05, P161=2.0859828575969366e-05, P162=1.5579329642816103e-05, P163=1.0340245131089029e-05, P164=8.536168316184997e-06, P165=6.880561546494377e-06, P166=1.3187441393976719e-05, P167=1.3064655927398951e-05, P168=1.8920545314335007e-05, P169=2.048296827213566e-05, P170=3.834034534572448e-05, P171=2.6106442950371475e-05, P172=5.140756176724695e-05, P173=4.868673449511092e-05, P174=4.985209487190823e-05, P175=8.441105424299374e-05, P176=0.00019750266659509665, P177=0.00020340648366646403, P178=7.974945384139816e-05, P179=4.857017914311829e-06),Row( P0=-426.994797776239, P1=-21.994860810073526, P2=-62.9543395877413, P3=-8.080812581196787, P4=-13.360881419230749, P5=2.9342668762120754, P6=17.582302728059386, P7=11.644704189736808, P8=-7.098392694147244, P9=-7.50446451898088, P10=-10.771933292845004, P11=15.076238520944571, P12=4.869030770692711, P13=-1.1325194415333597, P14=-4.6375967466641015, P15=-2.599823975652253, P16=8.910664364124534, P17=12.680891353522238, P18=2.752675355632729, P19=5.019976784577251, P20=5.269148323181993, P21=3.7988614028515624, P22=-1.4888823392997035, P23=5.32240467335434, P24=4.245033838442999, P25=-6.2112934499056385, P26=1.2158477717712874, P27=6.168526774165678, P28=0.49445967467997337, P29=3.509267854267885, P30=1.1795346067109544, P31=1.4288925140576787, P32=3.9127129904512494, P33=5.2177630020346015, P34=-0.5731410489188777, P35=4.492000534731455, P36=-0.5503897777182066, P37=0.8546348201902422, P38=1.7085957165750392, P39=0.8805277719947119, P40=0.5586876912635225, P41=0.4598114679540349, P42=0.3690201134032875, P43=0.32665453460361366, P44=0.34823730635868266, P45=0.3941145211367574, P46=0.4982110013377618, P47=0.6416803728011126, P48=0.6154889061888639, P49=0.5406019693849599, P50=0.49920890381244887, P51=0.515310664450837, P52=0.0026207453111206564, P53=0.0003148754319931607, P54=1.1371981049131352e-05, P55=2.69296677272388e-06, P56=5.257210542909414e-06, P57=8.745706633517745e-06, P58=6.231710498254044e-06, P59=5.920355265196969e-06, P60=1.7061618401056926e-05, P61=3.900276142437991e-05, P62=7.269407568668646e-05, P63=0.00010138558475781416, P64=4.595451008189135e-05, P65=6.418686193119162e-05, P66=0.00012933161678302702, P67=0.00019571494555746202, P68=0.00011308830292801904, P69=9.195071627717378e-05, P70=0.00010558945809541239, P71=5.252947681258502e-05, P72=3.88268475573577e-05, P73=5.1246941539512246e-05, P74=0.00010277047953003639, P75=0.00017514344129935302, P76=0.00033236504188731306, P77=0.0007034201635594781, P78=0.0008366132427011757, P79=0.00146889196705284, P80=0.0019263942626984805, P81=0.0019929180085091337, P82=0.004632454803043408, P83=0.008067708758988004, P84=0.00988642179042981, P85=0.01325869687743519, P86=0.0049689217345948115, P87=0.004820303187007445, P88=0.012488778349541025, P89=0.021075952739687934, P90=0.06051221843257894, P91=0.6483704655057537, P92=1.5778709791543826, P93=1.8650574194566223, P94=1.2248293125692185, P95=3.5806797448656065, P96=6.209568284454726, P97=6.505590142379049, P98=11.141133194256104, P99=13.11167408963609, P100=1.7276490884898517, P101=0.6829814022740646, P102=0.3151402033035998, P103=0.23153729645791934, P104=0.026057404157018214, P105=0.01262967990020944, P106=0.02300406660321488, P107=0.038325004509729706, P108=0.014741109714801616, P109=0.006801543690686213, P110=0.008696800775501591, P111=0.021762817265549635, P112=0.8146803866099616, P113=3.340501311801839, P114=4.331370261279144, P115=3.3965623833952803, P116=4.4793509258415805, P117=4.756651742643905, P118=1.7016622249738895, P119=1.1219401575025312, P120=0.8122263318765569, P121=1.2426138800695803, P122=0.19686776417102989, P123=0.01974530296749224, P124=0.13418104314506474, P125=0.743625029054486, P126=0.44126312306899085, P127=0.10945191531738538, P128=0.14745123391518056, P129=0.7992440895588769, P130=0.643834973160347, P131=0.1585137437448074, P132=0.0859466228788447, P133=0.04447088161378162, P134=0.07750908049642317, P135=0.023553611470262893, P136=0.04698510941932329, P137=0.2954897250611118, P138=0.5823634314602898, P139=0.32059414650718426, P140=0.13984530016706165, P141=0.16715722683391301, P142=0.062178499355419636, P143=0.03150657901552525, P144=0.07238193580438104, P145=0.15285168420668072, P146=0.06592488985248433, P147=0.04215496238085042, P148=0.016736430807052997, P149=0.007451884207119403, P150=0.005607774099196123, P151=0.003125159251885956, P152=0.0014494114822002839, P153=0.0028736447543146423, P154=0.009093600368465614, P155=0.0010195467888655305, P156=0.0015323946584555165, P157=0.0015304462798844175, P158=0.0034844562670540958, P159=0.0036209678752418947, P160=0.0026387928685778594, P161=0.0016529042856614818, P162=0.002127778960627583, P163=0.002151639455332119, P164=0.0015470940394082815, P165=0.0019157166794230946, P166=0.002236478882013314, P167=0.0018278920301535335, P168=0.0025637301492699976, P169=0.0021239570061816837, P170=0.0015567092450175656, P171=0.0009998222714515427, P172=0.000525818171051734, P173=0.0003547025865006256, P174=0.0004063130667986506, P175=0.0004478782963344616, P176=0.0005291703370285878, P177=0.00045288958351464583, P178=0.0001967948857262156, P179=1.794382580477632e-05),Row( P0=-745.9148099314265, P1=49.862747681242276, P2=-19.144728377899245, P3=-3.1426758753686928, P4=-17.199827471686007, P5=-19.727447544613728, P6=-4.230771178152203, P7=1.9751808495416272, P8=-8.998660624564414, P9=-12.968380924399018, P10=-4.102596844499775, P11=-0.7427846703120226, P12=-8.070271237747471, P13=-4.780547358582378, P14=0.4213233333182387, P15=5.076430909206391, P16=4.417092979250992, P17=11.523765325732935, P18=13.253773540237898, P19=18.940736486412032, P20=13.820852635217083, P21=12.919613188911557, P22=7.9314588653998035, P23=-0.8013953642024163, P24=0.3184260703109188, P25=-2.983928764532207, P26=-2.6575887836989907, P27=4.393030504473665, P28=0.4639060720513826, P29=2.780903968534797, P30=-4.772480855542881, P31=1.572656625965377, P32=-0.8480179493085472, P33=2.0288623122851743, P34=-3.6756266984251567, P35=-2.2865451481123715, P36=-6.760221422942245, P37=2.3824865203668266, P38=0.7050376367875674, P39=2.8316791999802713, P40=0.653311102487795, P41=0.6161728383740253, P42=0.5231643936460489, P43=0.479921627941512, P44=0.5211725213310086, P45=0.5002450892276584, P46=0.4875662986776718, P47=0.5642596166144984, P48=0.653049505582298, P49=0.7123465871697828, P50=0.6867204647584876, P51=0.6362756739851908, P52=0.0004951687200184654, P53=1.9498040940297693e-05, P54=6.78006217298415e-07, P55=2.2836861605807277e-07, P56=3.1353895072581495e-07, P57=1.902363183552569e-07, P58=9.415722506115564e-08, P59=7.939769550742614e-08, P60=6.356143614323112e-07, P61=3.092384169292403e-06, P62=0.00012749408648984087, P63=0.0008889014798513485, P64=0.0021002757062929312, P65=0.0046676681757695316, P66=0.0023412956630068926, P67=0.0017824699447181896, P68=0.003957935353404815, P69=0.002622686414419047, P70=0.0003877130935108145, P71=4.625296177173279e-05, P72=5.847199842332551e-06, P73=3.6578571294081885e-06, P74=1.2579710199607444e-05, P75=5.1500650622081864e-05, P76=0.00021364801629504638, P77=0.0011694054508371472, P78=0.0020368402183466522, P79=0.0014582971929851104, P80=0.000479683633920666, P81=6.690244180088135e-05, P82=6.006436470509716e-05, P83=0.00015082622017252885, P84=0.0013231668225351211, P85=0.006123390298507611, P86=0.003768093685716093, P87=0.0005088511154308066, P88=0.0002457414832399606, P89=0.0009307898382627473, P90=0.001168639887012698, P91=0.0005283093930070377, P92=0.0002087580211632169, P93=6.770253939753347e-05, P94=2.1274404639792403e-05, P95=1.6754057052677008e-05, P96=3.309956253708859e-05, P97=0.00020866543489433576, P98=0.0002825394901868134, P99=0.00016207811276415862, P100=5.898291946775955e-05, P101=5.092960366939077e-05, P102=2.532998790187175e-05, P103=8.757761077576118e-06, P104=8.346198375144864e-06, P105=1.479926423046119e-05, P106=6.205636961543616e-05, P107=4.0188875270707883e-05, P108=1.275899523891436e-05, P109=4.767594436386771e-06, P110=1.6994716668521966e-06, P111=4.602534783056551e-06, P112=1.1221592603851505e-05, P113=1.6546802399619615e-05, P114=3.985718045358479e-06, P115=4.252560343503091e-06, P116=1.0744558040810155e-05, P117=1.598450543145748e-05, P118=2.151812463084808e-05, P119=1.8406767142635806e-05, P120=9.684739960553105e-06, P121=1.3335029189492529e-05, P122=1.8107686397725186e-05, P123=3.1989608273220755e-05, P124=2.6651210777283986e-05, P125=1.2215796036598418e-05, P126=8.514391297407118e-06, P127=7.618839839364786e-06, P128=5.976868679995469e-06, P129=3.575643538033633e-06, P130=4.315371938002114e-06, P131=5.239429066582005e-06, P132=6.184634650684075e-06, P133=6.928031862622312e-06, P134=7.1513728691584074e-06, P135=1.462526856982915e-05, P136=4.613318483362145e-06, P137=6.574351260357699e-06, P138=4.425516888477667e-06, P139=5.389595165169981e-06, P140=6.880237072369724e-06, P141=7.16917624484645e-06, P142=5.310216878620927e-06, P143=3.7572048769054843e-06, P144=3.2502323432468592e-06, P145=3.1208984246041584e-06, P146=3.8085802123223613e-06, P147=6.128557874532224e-06, P148=5.082532136513126e-06, P149=5.657284529775723e-06, P150=3.674548851567699e-06, P151=1.4625708484117615e-06, P152=1.255583007778593e-06, P153=2.9647240887899327e-06, P154=2.628725123114871e-06, P155=3.504032301811983e-06, P156=7.053294513012762e-06, P157=1.1647189815495057e-05, P158=1.2493370763175592e-05, P159=8.098261714228884e-06, P160=7.897038066172194e-06, P161=5.314219829536528e-06, P162=4.188405861986292e-06, P163=3.2155611653758347e-06, P164=4.702225603848841e-06, P165=4.117215291052677e-06, P166=4.957223993962149e-06, P167=7.284677950457804e-06, P168=6.6932440904057425e-06, P169=6.012745709341914e-06, P170=7.4282745832720775e-06, P171=6.837424377902087e-06, P172=2.998254680385643e-06, P173=4.8704779967545425e-06, P174=1.1477680989719803e-05, P175=1.1587200072634205e-05, P176=1.0689022701509569e-05, P177=7.493667122635345e-06, P178=3.0006503137973214e-06, P179=1.8724187032017208e-07),Row( P0=-646.7133423923269, P1=28.174224361229687, P2=-40.36498797449464, P3=1.0120920333691517, P4=-23.07634677278981, P5=-20.531284170030446, P6=-0.09352872965096581, P7=-7.0000110927914845, P8=-5.471712805944732, P9=-13.295149518384592, P10=-7.235059199876506, P11=4.837784525794028, P12=-4.493391078463256, P13=8.182087284976149, P14=8.801306221357226, P15=7.158053834214256, P16=9.57838480769567, P17=5.407732896715438, P18=5.7969312531835975, P19=6.434477766926373, P20=3.99981239407771, P21=6.526238897854827, P22=-2.661302013332022, P23=-3.2722637907985384, P24=4.387836388178905, P25=-0.05715335613576309, P26=2.256074271680416, P27=2.612062796497194, P28=-2.0743202819717377, P29=2.367057318774974, P30=2.015081601238566, P31=3.4752153132594996, P32=0.8062025453045565, P33=0.48945632404146994, P34=0.39112958518214086, P35=1.6085902374566154, P36=0.49996247431149543, P37=0.1782618061372698, P38=-4.93613200666308, P39=0.8094553080221049, P40=0.690443404226519, P41=0.6665139094552207, P42=0.632506308519475, P43=0.5910975804199734, P44=0.5650978204129811, P45=0.5824059712221554, P46=0.5760404046566234, P47=0.538164266222905, P48=0.5087629864593197, P49=0.5572949901610355, P50=0.6156131328270237, P51=0.6712527775018114, P52=1.7636938245434943e-05, P53=4.207402514009576e-06, P54=7.098097933351332e-07, P55=2.598798474658625e-07, P56=1.0676062362057324e-06, P57=2.7831529253130055e-06, P58=7.90210044198356e-07, P59=1.9980313326933693e-07, P60=3.6497806195736326e-06, P61=2.489184685791865e-05, P62=0.00021647594956452701, P63=0.00037093990134514537, P64=0.000385282053651929, P65=0.0017657796195992737, P66=0.008035307447177679, P67=0.01458106095625892, P68=0.008426920340155263, P69=0.02019748454080361, P70=0.032958279914184804, P71=0.012669401759992052, P72=0.004145935250729493, P73=0.007402782103930257, P74=0.005135733540149235, P75=0.0008074299085285344, P76=0.00040558190554532554, P77=0.000503105561090322, P78=0.0008867746144206828, P79=0.0020009379564402195, P80=0.004328564022098153, P81=0.007544687742436437, P82=0.015035128071220103, P83=0.01187847331098857, P84=0.003054625393761407, P85=0.0019432508179932574, P86=0.0013623327944289553, P87=0.0061348059164914245, P88=0.03490062699604029, P89=0.04206078558928888, P90=0.01900985699682958, P91=0.00836538759045457, P92=0.006009434277416903, P93=0.011661832074700658, P94=0.019003460621293104, P95=0.030139781511931613, P96=0.014039860099750097, P97=0.007743187974240394, P98=0.004957321809656445, P99=0.002709329513336917, P100=0.08105717988415329, P101=0.19496018884548766, P102=0.027955860335715225, P103=0.004166057616880828, P104=0.024625894709456753, P105=0.0099612053181326, P106=0.0014822908601414269, P107=0.000686281733929037, P108=0.00023176031526850042, P109=0.0019237335983485475, P110=0.0023025106403998614, P111=0.0021006194371934227, P112=0.002039950063443167, P113=0.002077246346784542, P114=0.0005077210797820305, P115=0.0006997256643510653, P116=0.06139632891185952, P117=0.03309127740351017, P118=0.00452088497183848, P119=0.0014007486589588467, P120=0.00029000175778690475, P121=0.011842044721123601, P122=0.017778183424074178, P123=0.0017617373689075286, P124=0.0002769434787102611, P125=0.0028489665235015217, P126=0.002020146596146361, P127=0.0012923032952796153, P128=0.0012717713445260799, P129=0.0031286983708676197, P130=0.0005687190376731548, P131=0.0010111273253158404, P132=0.000603629709351678, P133=0.00019850216776569847, P134=0.004277024939264108, P135=0.0040309975119300154, P136=0.0005941490913727995, P137=0.0011370061900445923, P138=0.0008345329467984734, P139=0.00030656814294655754, P140=0.00036768019467201397, P141=0.00018005652378782577, P142=0.00015289222860893873, P143=0.00038220081584451995, P144=0.00029433543990472496, P145=0.0005931316911989462, P146=0.000867129348189698, P147=0.0005269024964812629, P148=0.0007632027592683647, P149=0.0002890512484931052, P150=0.0002100035035778917, P151=7.101243681738069e-05, P152=6.224493814058205e-06, P153=4.867777253718466e-06, P154=2.4204193821856505e-06, P155=2.3280202009238122e-06, P156=4.728650678325259e-06, P157=8.74343478889765e-06, P158=1.1820960727363793e-05, P159=1.6883454418785716e-05, P160=2.551376407363105e-05, P161=1.9509559625301133e-05, P162=1.7571631368673392e-05, P163=1.0199371673759405e-05, P164=1.1413064470427263e-05, P165=1.6723998476083656e-05, P166=1.588348972199879e-05, P167=1.2443164868837676e-05, P168=3.180878428832016e-05, P169=3.0207636697517602e-05, P170=3.973993873837763e-05, P171=7.000972800080752e-05, P172=0.00010066488432288949, P173=6.317101157704149e-05, P174=0.00010720100938392187, P175=0.0001525989679201676, P176=0.00017830893356236927, P177=0.00015718144726305482, P178=7.554861900044914e-05, P179=8.493542360843402e-06),Row( P0=-677.0796891606665, P1=28.238067686138056, P2=-44.5505841691757, P3=-1.6553984032567626, P4=-27.101581101998296, P5=-20.30546345390873, P6=-4.972667400923236, P7=-8.103830546597266, P8=-8.728715985994189, P9=-15.29960857415157, P10=-6.869679110760978, P11=3.0551308646294757, P12=-5.664080759199356, P13=10.676461415811701, P14=7.29389896604633, P15=8.84346526122293, P16=11.720365205217817, P17=7.803927853832712, P18=10.386910409778205, P19=11.761180865963482, P20=7.312791020088601, P21=10.8623965595737, P22=0.9507311695813945, P23=-0.5012527434446747, P24=1.7986471067997825, P25=-2.2980362304944166, P26=3.6129839147338263, P27=1.6950000998178936, P28=-0.43972604083639455, P29=2.2935303084078074, P30=-1.0520595912168422, P31=3.1395176105167435, P32=-0.1723855784342798, P33=1.4993639539421144, P34=-2.6133137374964766, P35=0.8636743481248107, P36=1.2115286342134237, P37=1.6066602172784863, P38=-5.575220713219162, P39=-0.8077927618168425, P40=0.6481412198916697, P41=0.7065481863478287, P42=0.6781506561755943, P43=0.5857924061449495, P44=0.5987239039075763, P45=0.6087234697113508, P46=0.5732043123976219, P47=0.5710125832685979, P48=0.5748723251667429, P49=0.6269806430555227, P50=0.6548973491065085, P51=0.666441984269229, P52=1.9182284241181805e-06, P53=2.058683077115378e-06, P54=7.2498455763223e-07, P55=5.7154479110691034e-08, P56=8.597193251382856e-08, P57=1.1756435198638909e-07, P58=4.541448507671158e-08, P59=3.211609754807917e-08, P60=1.8526863179176995e-07, P61=2.197737229560258e-06, P62=9.911655669017443e-05, P63=0.0006023053235512649, P64=0.0007644533463949285, P65=0.002028940843014495, P66=0.005297417015345879, P67=0.0072750392109074335, P68=0.00826399806886723, P69=0.021972265872009674, P70=0.01876260459541178, P71=0.007471805849014446, P72=0.0035346737696147503, P73=0.005957776418618205, P74=0.0058217300880307925, P75=0.0017957146679233253, P76=0.0010330482123125843, P77=0.0009949518258447023, P78=0.0014958668495441089, P79=0.003218552795267005, P80=0.0064703507444139425, P81=0.010017778644080051, P82=0.009120476072546683, P83=0.0027995723720441344, P84=0.0008908986162286913, P85=0.0015449160864321116, P86=0.0038725485959517215, P87=0.013829880139207653, P88=0.02162101883719407, P89=0.01912460324222883, P90=0.012286538972359408, P91=0.011263332524300182, P92=0.012867344705879727, P93=0.019321431390298783, P94=0.013943557193328458, P95=0.011852858661552893, P96=0.016646279775693478, P97=0.007165012876150246, P98=0.004123667695385193, P99=0.029563218089807528, P100=0.027681114049477797, P101=0.009926318619712264, P102=0.02405639675760563, P103=0.012390099303438366, P104=0.00653458981672459, P105=0.0017524846246281826, P106=0.0005218754020043254, P107=0.0010035981009968754, P108=0.004973709397896745, P109=0.008427522111532068, P110=0.0018102940448118372, P111=0.0005398272459757043, P112=0.000571045384704158, P113=0.001306121390096704, P114=0.004873937009132596, P115=0.01340242396222434, P116=0.00824346836240323, P117=0.004016363752420771, P118=0.0019395022567620186, P119=0.0055917403376176894, P120=0.008305196080257874, P121=0.004478309814792937, P122=0.0013443430714955467, P123=0.0014195507185740448, P124=0.001960874633956498, P125=0.0011090999223594342, P126=0.0008013170370139653, P127=0.0014271287293885035, P128=0.001000909311725859, P129=0.0009086796778372589, P130=0.0008194427961792039, P131=0.0006112250614103856, P132=0.0004828319248862996, P133=0.0012807473843281427, P134=0.0004941122767971582, P135=0.0002973940260237056, P136=0.0006464058184070045, P137=0.0003541201642983684, P138=0.00026213061790236785, P139=0.0004278543180820847, P140=0.00013932443263383568, P141=0.00010848036034478417, P142=9.305082178826196e-05, P143=6.18379111680415e-05, P144=0.00012357374538487815, P145=0.00020244039724902474, P146=0.0004103799544897747, P147=0.0007569298444755901, P148=0.0007771534214006306, P149=0.0008160488250867918, P150=0.00024660210359000404, P151=0.00017554543978846423, P152=3.076339092062593e-05, P153=1.2355912785273303e-05, P154=2.5456386734799863e-06, P155=2.0969244226869827e-06, P156=3.6643339586783683e-06, P157=9.406087573975135e-06, P158=1.1854681484790402e-05, P159=1.501153127039544e-05, P160=3.532546264344468e-05, P161=6.782030069465452e-05, P162=2.7425104439299385e-05, P163=1.581526707299211e-05, P164=1.57283289092463e-05, P165=2.3145763355340584e-05, P166=3.133730300367724e-05, P167=4.034178825256216e-05, P168=3.8932979557223034e-05, P169=4.173926761142172e-05, P170=7.45771964159514e-05, P171=0.00018077280853584545, P172=0.00014999814495171338, P173=7.642029044881274e-05, P174=6.0832449778895535e-05, P175=0.0002856742624760358, P176=0.00040176561727929517, P177=0.00020512438880874662, P178=5.522152171711148e-05, P179=3.0437323887995838e-06),Row( P0=-785.4046636090583, P1=40.592251157041815, P2=-10.632034529867077, P3=-16.216493570455846, P4=-12.703867592810228, P5=-11.113292376913336, P6=-9.450178273026157, P7=3.000968091243538, P8=-9.098411184358604, P9=-8.264661566565763, P10=-7.000057503972108, P11=-4.646080947982038, P12=-7.836826648211701, P13=0.055515487671796135, P14=1.5345999226403686, P15=10.821240018083035, P16=13.222917789141114, P17=9.03788461501556, P18=11.008053043210293, P19=6.847341375742055, P20=3.062487089513276, P21=4.090563706644056, P22=-2.426973816168655, P23=-0.7126931194690957, P24=-0.49504261089250035, P25=-0.6149984798922697, P26=-2.567951625601665, P27=0.05782362033598112, P28=0.11215272456709348, P29=-0.5049674404654085, P30=-0.18398372622761094, P31=0.5098685892139718, P32=-1.028979666544174, P33=1.5588032906309734, P34=-3.384554442091097, P35=-1.238479805299879, P36=-1.7938021016300982, P37=0.34098767747526215, P38=-2.6152512501972436, P39=-2.2422437387633507, P40=0.6374218891966376, P41=0.6407626044196021, P42=0.5965086278018571, P43=0.5905686839903799, P44=0.5953814435726434, P45=0.5750215814868658, P46=0.5570416104464428, P47=0.5821391302431403, P48=0.6057530189816513, P49=0.6221231347064363, P50=0.6737913934304571, P51=0.6743771369249469, P52=1.5524565639415267e-06, P53=4.2374266724237965e-06, P54=1.8284388956878402e-06, P55=3.454308709264838e-07, P56=6.414898584193047e-07, P57=8.364775839547026e-07, P58=3.9429882394928295e-07, P59=3.47426926657978e-07, P60=2.4815893778670263e-06, P61=9.34334200431782e-06, P62=6.6424310082035e-05, P63=0.00015205276417492518, P64=0.00020050637341047806, P65=0.0016737740683578398, P66=0.004839550739090713, P67=0.00783716549133798, P68=0.004145450384746272, P69=0.0023433269156630104, P70=0.0011224495614406992, P71=0.00018301166834320818, P72=1.6851143093721393e-05, P73=1.0354477659339253e-05, P74=2.4517979815528764e-05, P75=4.778278310073956e-05, P76=7.320254353336002e-05, P77=0.00035289081550464587, P78=0.0010356658006129673, P79=0.0020499843984112423, P80=0.0036843778966468305, P81=0.003926765050622212, P82=0.0021526245593627143, P83=0.0010639805333370776, P84=0.0009589313067006287, P85=0.003433305009032476, P86=0.004064179503635825, P87=0.0016002752239969882, P88=0.0007646703470268423, P89=0.0008729252863922393, P90=0.002440190236259253, P91=0.003144060177014038, P92=0.000746788408651753, P93=0.0008207233881715414, P94=0.0003507774683670188, P95=0.00022008054601966253, P96=0.0002549027594742497, P97=0.0002577903576844046, P98=0.0005773362627880309, P99=0.002349067508585137, P100=0.0003950675372642125, P101=0.0004870946137230336, P102=0.0006853932030748914, P103=0.00026299541380188585, P104=7.134397414661226e-05, P105=4.54072496073122e-05, P106=9.28034301428414e-05, P107=7.959915462431525e-05, P108=9.489928273384382e-05, P109=3.322406000880588e-05, P110=1.4668000693405698e-05, P111=5.176192139983597e-05, P112=1.3012280071094177e-05, P113=2.605961525001822e-05, P114=0.0001275223161589848, P115=0.00013612113940865918, P116=5.438686728214941e-05, P117=3.51520958372466e-05, P118=6.355125694656787e-05, P119=7.247410039917787e-05, P120=2.6577776965994438e-05, P121=1.5387039134776034e-05, P122=2.108238799329997e-05, P123=3.2823614913983496e-05, P124=2.1403325052094048e-05, P125=1.1843565360632902e-05, P126=1.1676668206184327e-05, P127=8.966328438875264e-06, P128=4.492920123027376e-06, P129=6.795045434603915e-06, P130=6.7935738572468745e-06, P131=4.858840183492328e-06, P132=2.393300680671431e-06, P133=5.997958836943525e-06, P134=5.509489816100975e-06, P135=5.2242371569311835e-06, P136=5.528323946561465e-06, P137=2.3527926287460942e-06, P138=1.8218365790544042e-06, P139=2.080756761636137e-06, P140=3.762108840612116e-07, P141=3.2432025932154326e-07, P142=3.5500623948576097e-07, P143=3.4700702797557115e-07, P144=5.212075672333731e-07, P145=7.389345842438843e-07, P146=9.528495154049224e-07, P147=1.4355566717289001e-06, P148=1.7409883255628587e-06, P149=1.8453256755943628e-06, P150=3.3246678710289056e-06, P151=3.1046766420445555e-06, P152=4.523709618408716e-06, P153=1.0383811884310161e-05, P154=5.53445638120614e-06, P155=2.924367308099161e-06, P156=3.15514730532961e-06, P157=3.883997182029685e-06, P158=5.374965209294548e-06, P159=1.894244701568316e-05, P160=3.921313935906213e-05, P161=9.331349385712526e-05, P162=8.795898779280266e-05, P163=4.944713794649964e-05, P164=6.628282385493329e-05, P165=8.515695749038554e-05, P166=5.764780912827387e-05, P167=3.815297322581012e-05, P168=2.9202548445131262e-05, P169=1.3911464601528795e-05, P170=1.5255342052113827e-05, P171=1.5642898824618992e-05, P172=1.0950595526535573e-05, P173=4.6703267832708255e-06, P174=5.909294946159062e-06, P175=1.1683776746712777e-05, P176=1.5290446044140734e-05, P177=1.5857680424623574e-05, P178=7.8611875091294e-06, P179=9.087341900697044e-07),Row( P0=-681.0161089019145, P1=37.39129906114205, P2=-21.55068371766759, P3=-4.167708821043195, P4=-16.09097964291075, P5=-19.292843301457278, P6=-4.399691318103462, P7=-7.026200994690357, P8=-12.533294307569522, P9=-15.511677809347882, P10=-8.094578969390312, P11=2.2831021602055737, P12=-0.06643827031314796, P13=11.158980049918481, P14=13.863863147798941, P15=14.773110574812334, P16=10.129380180921359, P17=8.091373315924198, P18=4.158851459365819, P19=5.521592962531852, P20=5.416698216742383, P21=8.966278377124446, P22=3.1809575143114217, P23=-1.8302391063467276, P24=1.188955027507541, P25=0.9383490161604409, P26=4.486597129234597, P27=4.566897940998017, P28=-0.2864528789215485, P29=1.6137694846460846, P30=-1.1063774519694076, P31=2.4674225243125587, P32=-0.685520575566447, P33=0.3087896145716556, P34=-3.293130257278413, P35=0.9041840699533227, P36=-0.5300917789232772, P37=1.6811090267934943, P38=-4.043442806271246, P39=-0.6027120026762789, P40=0.7100398316875417, P41=0.731251743531652, P42=0.7441471388862423, P43=0.7126835168863639, P44=0.6437984481071056, P45=0.6074059136030521, P46=0.5649996083496508, P47=0.5348366386665052, P48=0.566539372885131, P49=0.636077283492964, P50=0.6816152560401929, P51=0.7052361795930533, P52=2.698648292835505e-05, P53=8.696551294410636e-06, P54=2.587964460860512e-06, P55=7.014958816188769e-07, P56=6.040984855972961e-07, P57=1.0404952347955587e-06, P58=4.934638781802499e-07, P59=3.014684420702443e-07, P60=5.347481782919776e-06, P61=4.8284608282501795e-05, P62=0.00045784975957253115, P63=0.0014325027796196982, P64=0.0011842340925902392, P65=0.002809300637754076, P66=0.010009319950570634, P67=0.01966163356108456, P68=0.017910229432949648, P69=0.04148275422524318, P70=0.056199509924963746, P71=0.0328061042771854, P72=0.015925023221711014, P73=0.007607618668610468, P74=0.0028619944998877667, P75=0.00077283977849065, P76=0.00025319916202154924, P77=0.000360482306663558, P78=0.0004366962690411036, P79=0.0005896117043895637, P80=0.0008061720216610091, P81=0.0010786667037869845, P82=0.0019227359137631728, P83=0.0021398003913165827, P84=0.002112490475898885, P85=0.0034591720168960683, P86=0.0034174399884620245, P87=0.006561507188046072, P88=0.015079047096553558, P89=0.012532450229503705, P90=0.009102006745939197, P91=0.009810520778886682, P92=0.01470336840077488, P93=0.008778920564959403, P94=0.0029721416651741018, P95=0.001953295201695918, P96=0.0015192758725219619, P97=0.0008521865296962818, P98=0.005539668642171203, P99=0.018157032288350455, P100=0.008536795352935597, P101=0.015144345559457515, P102=0.014144044762600692, P103=0.003638913371150617, P104=0.00030819134099059647, P105=8.078455691406405e-05, P106=0.00012019029789360352, P107=0.0007202476376672134, P108=0.0017383510089532908, P109=0.0011495524025645744, P110=0.0006001793948019336, P111=0.0005618329625463129, P112=0.0005708137516341137, P113=0.00026733650907522535, P114=0.0003989874950540481, P115=0.0006400709103780375, P116=0.0007052542811038047, P117=0.0011075660292979425, P118=0.0008198524517791021, P119=0.00017991113196568395, P120=0.0007220612802591668, P121=0.002437086872092435, P122=0.0017187856810712344, P123=0.0004999947064222925, P124=0.00029860775636082504, P125=0.0014503321737198206, P126=0.0005173986668814802, P127=0.00013925436772509506, P128=0.00016982841507889634, P129=0.0003963334720158315, P130=0.0002454450550451486, P131=5.540247351697748e-05, P132=0.00017402487166688082, P133=0.0003527179255531916, P134=0.00019957001316911797, P135=0.0001366626844467572, P136=0.00016070691806629474, P137=0.00010659722939089584, P138=0.000130093311876929, P139=0.00012132268709968337, P140=6.993312654034493e-05, P141=4.090858279546893e-05, P142=2.716268972463954e-05, P143=1.4442777344385465e-05, P144=2.4789241251465175e-05, P145=1.997868046380267e-05, P146=5.050838427110392e-05, P147=7.884630176659066e-05, P148=4.6645450332402375e-05, P149=6.559728898752437e-05, P150=4.539845395977694e-05, P151=3.7331121107605165e-05, P152=2.340303805568935e-05, P153=1.2874719153154628e-05, P154=1.0803102441860499e-05, P155=1.214943309203229e-05, P156=2.714948885406093e-05, P157=5.700717919914361e-05, P158=9.449273370400234e-05, P159=9.486835503667286e-05, P160=7.196230742656775e-05, P161=8.547610002439142e-05, P162=0.0001047487197653958, P163=4.5304507378675815e-05, P164=2.7896194023019424e-05, P165=2.532138675783141e-05, P166=3.7108811255283075e-05, P167=7.38603239278793e-05, P168=7.994211955603115e-05, P169=8.20981828600744e-05, P170=7.904389815736294e-05, P171=7.882426947179396e-05, P172=9.654999695270893e-05, P173=0.00010346213969422471, P174=0.00011982008627943711, P175=0.0002993713862748095, P176=0.0003805818299552962, P177=0.0002817115063563331, P178=0.00019879179370763027, P179=1.8401118449246077e-05),Row( P0=-739.9411385858115, P1=42.55068173518993, P2=-14.031288616126073, P3=-1.4504880934461926, P4=-16.91990784937054, P5=-8.11208543477091, P6=-5.459469640520316, P7=0.32603767564398883, P8=-6.057551226491064, P9=-10.049591193717301, P10=-6.1307579362820475, P11=-0.35790867428409473, P12=-6.649335062457253, P13=1.5507384990953699, P14=2.3243557389939933, P15=6.220407496397103, P16=7.0056407891081065, P17=7.598641582262448, P18=13.277302420931196, P19=10.18517584067502, P20=7.182151708863684, P21=9.792324836436533, P22=0.1847059903007086, P23=1.1895634528580534, P24=2.7392377977468687, P25=0.8010051701610009, P26=0.7642052300295588, P27=2.3104083963506885, P28=-0.36180065364578695, P29=1.1412331754630936, P30=0.6033204901497248, P31=1.6548047851560097, P32=-1.241508821008091, P33=1.8777683198133883, P34=-2.8247249049436425, P35=0.20799037018998465, P36=-2.3144063524587106, P37=-0.8180446974565968, P38=-3.7070082246514717, P39=-0.9104332447337397, P40=0.7219727078044301, P41=0.6725397926898985, P42=0.6266802308134958, P43=0.5828916652691001, P44=0.5729770798235544, P45=0.6120192038405056, P46=0.6016426593945394, P47=0.61156618232047, P48=0.5972688569258736, P49=0.597856747270701, P50=0.6866578107423738, P51=0.7211861977491492, P52=1.6547077801236637e-05, P53=8.416475697194769e-05, P54=4.9662712901185974e-05, P55=1.320816307685856e-06, P56=1.104405585506373e-06, P57=1.6104957124453583e-06, P58=6.598359951000482e-07, P59=5.083782032108542e-07, P60=5.841926421760197e-06, P61=2.086328129126023e-05, P62=0.00014938698853255618, P63=0.0004240396276211277, P64=0.00043234562876358335, P65=0.00159536940774943, P66=0.004074556050530994, P67=0.007390242598765871, P68=0.003287943970076794, P69=0.0014743093856388612, P70=0.00157464018960883, P71=0.000790453930092872, P72=0.00019738410361362811, P73=4.47051238414754e-05, P74=2.671605095262214e-05, P75=5.360272436142325e-05, P76=8.21139419163857e-05, P77=0.0001629002400358275, P78=0.00027639383969933523, P79=0.0006977929912673368, P80=0.0014393478046849182, P81=0.0007027594682956323, P82=0.0003219217845109671, P83=0.0002530014193584171, P84=0.0001811942528497701, P85=0.0002689996547170374, P86=0.000241970436564992, P87=0.00041857189174452915, P88=0.0008014720341667515, P89=0.0005784025377854398, P90=0.0004933171375375381, P91=0.000296685143220745, P92=0.00031522962346049654, P93=0.0002503938481202788, P94=0.0002446680452610349, P95=9.484649537314258e-05, P96=5.643431579745944e-05, P97=7.15856396937476e-05, P98=3.119897328657268e-05, P99=2.0444900741316007e-05, P100=1.5202268627685874e-05, P101=5.616532609329819e-05, P102=5.141357264057289e-05, P103=3.668148923522281e-05, P104=1.8176047030504386e-05, P105=1.4580671169287095e-05, P106=2.554393711404936e-05, P107=3.0755924280425043e-05, P108=2.3164275720438523e-05, P109=1.6437354420995247e-05, P110=1.0199123510003731e-05, P111=9.74458205738774e-06, P112=1.516069458812119e-05, P113=3.4875142436034823e-05, P114=2.87403725761135e-05, P115=1.8885067011591327e-05, P116=1.9433879049991525e-05, P117=2.410461372452534e-05, P118=1.7047005507305767e-05, P119=2.0131939340242326e-05, P120=9.2350619126592e-06, P121=9.857580241187494e-06, P122=1.7718246878842282e-05, P123=1.454055015404968e-05, P124=1.1354181229733186e-05, P125=1.0449657798633722e-05, P126=1.2663910468937634e-05, P127=9.96926562680842e-06, P128=6.200967595293901e-06, P129=8.242735126953892e-06, P130=4.862671051950476e-06, P131=3.5703981933474803e-06, P132=5.8674741279118665e-06, P133=6.891227461869841e-06, P134=6.734110860698092e-06, P135=9.221995524744609e-06, P136=6.821120366087987e-06, P137=7.557821988585991e-06, P138=5.367425203605777e-06, P139=4.630591953632972e-06, P140=5.9847236527267385e-06, P141=5.827166760231082e-06, P142=1.0652018799875303e-05, P143=9.614805930297428e-06, P144=1.2004280787404486e-05, P145=9.791027546677746e-06, P146=1.0491559210145541e-05, P147=2.1462536244122155e-05, P148=4.3085736732054444e-05, P149=3.331453105057855e-05, P150=3.843528657907079e-05, P151=3.736063290994149e-05, P152=3.272531059633049e-05, P153=9.625603326003206e-06, P154=4.689426994713724e-06, P155=6.382907749845944e-06, P156=1.6684333629145505e-05, P157=1.3357932169405538e-05, P158=1.6926742961230563e-05, P159=2.115816546642759e-05, P160=1.7107497890695355e-05, P161=1.424635447152331e-05, P162=1.1856741841112743e-05, P163=1.7290078885757507e-05, P164=1.3013599670992813e-05, P165=1.0715475057183515e-05, P166=8.567128200797406e-06, P167=1.248780627413099e-05, P168=1.1314597856808157e-05, P169=9.941513190098598e-06, P170=1.1839689816660615e-05, P171=6.154022937896015e-06, P172=5.387451852319699e-06, P173=4.761324617397614e-06, P174=6.494319571186433e-06, P175=5.793895640193271e-06, P176=8.530701794646403e-06, P177=6.099300338247225e-06, P178=2.0006877169596016e-06, P179=2.4504264754861424e-07),Row( P0=-388.0798685889985, P1=-18.531975044011915, P2=-63.83600352182231, P3=-10.78168534326458, P4=-5.39271645287508, P5=2.105587056000072, P6=16.816408331051104, P7=10.896839607323422, P8=-9.509015870598498, P9=-6.015388888444134, P10=-6.352692823874992, P11=20.10925886445096, P12=1.0594890773955217, P13=-2.749283475958794, P14=3.4603038600475395, P15=-5.232285861596219, P16=7.24906735983148, P17=6.794408586953808, P18=5.163497554868749, P19=5.809390983538407, P20=2.9794058361355744, P21=6.69152571574582, P22=0.733720946317944, P23=7.346940277647881, P24=7.927980696077727, P25=-6.504145989601739, P26=1.0842841257084017, P27=6.788379149925068, P28=1.1938113129356203, P29=5.842153914233523, P30=0.4457992730708819, P31=1.291593781247243, P32=3.30489931664349, P33=3.9758913078017555, P34=-0.47667072879955447, P35=7.37319328094591, P36=-1.4333292187741826, P37=-2.934812302687352, P38=3.5035630518017817, P39=2.2336192025870525, P40=0.6657232805547374, P41=0.4572438321918944, P42=0.36095185226886634, P43=0.348296533555945, P44=0.3206630206752317, P45=0.3416781481138988, P46=0.49177649703093446, P47=0.6707912913272017, P48=0.6506903874384578, P49=0.505453068387279, P50=0.5055132882622301, P51=0.6223757632113653, P52=0.006219963140717204, P53=0.0005162602546981869, P54=5.35402480262018e-05, P55=5.987621909369783e-06, P56=2.4278904380426105e-05, P57=3.7248697563021865e-05, P58=2.2453793185542814e-05, P59=3.968620860775622e-05, P60=6.922793210009487e-05, P61=9.728015465958064e-05, P62=0.0002259339172502715, P63=0.00029174396079466726, P64=0.00016236350383902058, P65=0.000189008786864161, P66=0.00031992968386239905, P67=0.00045525797276378395, P68=0.00021573766359837856, P69=0.0001596712820848988, P70=0.00014272345517221183, P71=0.00013282046292305912, P72=0.00019973699438488845, P73=0.0002339666372761207, P74=0.000578177655766335, P75=0.0006341827807756104, P76=0.0006162288616008753, P77=0.0011933473295653249, P78=0.0019005662247443577, P79=0.002739411528250846, P80=0.003038677716126145, P81=0.004666540607990064, P82=0.009621583945525333, P83=0.007286807838108596, P84=0.006008523092732247, P85=0.009502035807755032, P86=0.008802508337864685, P87=0.01744684351336224, P88=0.035761865125304215, P89=0.04081240077986982, P90=0.04588310654826675, P91=0.12035957972054362, P92=0.1970241315348783, P93=0.3001339916347673, P94=3.239446223022644, P95=10.122612672166738, P96=7.889108501901545, P97=14.272648912637813, P98=25.385490459154845, P99=19.83831513968361, P100=1.6171433313589443, P101=1.4714173747862633, P102=0.7417140828287586, P103=0.5787842032900749, P104=10.04536524599655, P105=7.835565959946801, P106=5.030969234189055, P107=2.7183324527520094, P108=0.39524108861406293, P109=0.22569213018229503, P110=0.09160298238314872, P111=0.23087282340749848, P112=0.22195970599691958, P113=0.18123083882656868, P114=0.3454064015277388, P115=1.5405079888811777, P116=13.338714502523878, P117=7.978839088056956, P118=4.171246944582199, P119=3.3272341850979896, P120=1.1424506899149869, P121=0.825038299686434, P122=0.4598000413756818, P123=0.22627870173532208, P124=0.36691241430019833, P125=0.7989453644865517, P126=1.2497686845641336, P127=0.27722457210669527, P128=0.18788932644071157, P129=0.18835086380207214, P130=1.6031963777617635, P131=2.760638070734306, P132=0.45201961207122504, P133=0.05365416753324897, P134=0.03636344261872108, P135=0.04579321630163737, P136=0.10092942455619686, P137=0.33544782022168274, P138=0.20430835911151302, P139=0.3890352149028556, P140=0.40586075188523213, P141=0.21773347554894407, P142=0.017650519705894915, P143=0.020477831773679463, P144=0.1231844893765008, P145=0.07796050897743004, P146=0.24119512831959064, P147=0.21538110877135172, P148=0.05615879012614337, P149=0.003931185625065648, P150=0.0029741711745897917, P151=0.01085115451554365, P152=0.008437569474918579, P153=0.0028140156613186635, P154=0.0010943029791643263, P155=0.0005218300177031849, P156=0.0009561586363408581, P157=0.001371963123724365, P158=0.0021336804373026795, P159=0.0032497561221374654, P160=0.0024234965797167067, P161=0.002118627147138089, P162=0.001977888877357527, P163=0.0016399438963124037, P164=0.0020983499787198333, P165=0.0026700553042349248, P166=0.003770486947495132, P167=0.0032710270089029857, P168=0.003372300266966108, P169=0.003758994057504202, P170=0.0027530180646174415, P171=0.0019480555194900386, P172=0.0015934202027784589, P173=0.0019450691923463025, P174=0.0021691388613338616, P175=0.0016642009163338643, P176=0.002640122550089671, P177=0.003631664846847523, P178=0.0022957858882300657, P179=0.0002381894387376364),Row( P0=-690.1944683533288, P1=27.58973617325566, P2=-38.91669617780942, P3=-9.234985143561872, P4=-22.466362334151345, P5=-10.763762615060694, P6=-3.9845688871315508, P7=-6.8726984016344455, P8=-10.447455166803472, P9=-11.412240486883261, P10=-9.034987031618723, P11=-0.9749427783767408, P12=-0.5096815936661107, P13=10.992994429281929, P14=9.956822418843029, P15=17.89642280965718, P16=9.368392515190251, P17=5.336754463092555, P18=4.445846612576141, P19=-0.9057803255437769, P20=1.6705876240280364, P21=0.39333212559516906, P22=1.0152754811685387, P23=1.0293440600981998, P24=-0.6205868479207375, P25=-1.3227029530627423, P26=-0.5921052949292759, P27=0.32324263752752413, P28=1.5392470068691035, P29=2.1671369725369343, P30=-0.8533273122302467, P31=1.110267665431575, P32=-0.4089314873737135, P33=2.199541580659664, P34=-3.8482949814901315, P35=-1.3666537851865148, P36=-1.816828748362034, P37=0.8028707695113472, P38=-2.2091267765093456, P39=-2.598116518178861, P40=0.5611981232808918, P41=0.6076664269862293, P42=0.6213169024706404, P43=0.5361444706355313, P44=0.5572742772680029, P45=0.5825368002387054, P46=0.5269054116044233, P47=0.5287031003416116, P48=0.5257176702175079, P49=0.5035004122382554, P50=0.4866538480091701, P51=0.5387682471360148, P52=1.4408788686987984e-06, P53=2.9510937696612106e-06, P54=1.5452478536732532e-06, P55=8.985455189761268e-07, P56=1.3352845391900722e-06, P57=5.651652399523235e-07, P58=2.1229221725058273e-07, P59=1.9224839266288222e-07, P60=3.8367953623226847e-07, P61=1.1643853112521939e-06, P62=8.797818257279563e-06, P63=3.2992568698817915e-05, P64=5.524022176164731e-05, P65=0.00031011320440276326, P66=0.002494873041336542, P67=0.006884866765302688, P68=0.006188647601725155, P69=0.014009370804053416, P70=0.012800029584541578, P71=0.003629500524843295, P72=0.0030690408950521803, P73=0.002890529667234723, P74=0.004442657609004538, P75=0.0022682858717178386, P76=0.0003695299623012881, P77=6.271255184243672e-05, P78=6.577005998801515e-05, P79=0.00010475359396112118, P80=0.00015770883768064412, P81=0.0003448910130538166, P82=0.001323935637539404, P83=0.0021807449901946685, P84=0.00342032845986873, P85=0.022240328478383208, P86=0.03284410634179203, P87=0.031198432001954064, P88=0.022732184454539676, P89=0.006195506144783552, P90=0.002975387628210434, P91=0.006462246319119261, P92=0.022470718608869993, P93=0.011331356617235756, P94=0.011623442272275856, P95=0.03724823413529949, P96=0.02641099619092226, P97=0.012240644090920159, P98=0.018290564034580646, P99=0.02564496452577987, P100=0.032067983193392845, P101=0.004913215978267542, P102=0.004321126809468693, P103=0.0049891342677667195, P104=0.0026287769975885823, P105=0.001517720034283708, P106=0.0037467921476556374, P107=0.007034271630219666, P108=0.004437863685565438, P109=0.0027981527206235885, P110=0.0005460586274302115, P111=0.00037726362865095, P112=0.00033323901303438096, P113=0.004727126306242822, P114=0.004681575958772083, P115=0.002156698011749789, P116=0.0005241706965609298, P117=0.00035314985695908565, P118=0.0004762922139783824, P119=0.001100499499897068, P120=0.0012198648809651194, P121=0.0039848132136173706, P122=0.0013364031273584865, P123=0.0002877439990415361, P124=0.0003055312315764391, P125=0.0005170953479264532, P126=0.0015443001264660078, P127=0.000710233210883859, P128=0.00025682387622558134, P129=0.0008087465760243335, P130=0.0011410726528011616, P131=0.00030041625081382437, P132=0.00035788257713225454, P133=0.00041911638829471217, P134=0.0003326803573762971, P135=0.00010893989652098644, P136=0.00015596701959141483, P137=0.00017362150305267137, P138=0.0001977023669758805, P139=0.0004543413232968586, P140=0.00031623153450486847, P141=6.693576757247534e-05, P142=0.00010263722554550664, P143=5.699631934967587e-05, P144=6.602075021863001e-05, P145=4.214185432503287e-05, P146=2.2687483578737727e-05, P147=3.327756734029716e-05, P148=2.8231523026441186e-05, P149=9.274927610013667e-05, P150=0.00010820406910062704, P151=6.968889924091829e-05, P152=0.00016719260891689174, P153=9.29548107824164e-05, P154=4.6000715537954503e-05, P155=3.197951516591511e-05, P156=8.662135155543019e-05, P157=5.140997876011501e-05, P158=2.3456817905835203e-05, P159=1.8965707315459355e-05, P160=1.4031392749939935e-05, P161=9.82179756177367e-06, P162=8.036491843362068e-06, P163=6.087319590156049e-06, P164=1.2209312903457803e-05, P165=1.6487583068204893e-05, P166=1.7856094921366557e-05, P167=2.0720171006988038e-05, P168=2.4165530676315662e-05, P169=1.9880974225308518e-05, P170=1.0567644046905195e-05, P171=6.980154486209657e-06, P172=4.771778013863184e-06, P173=3.3664081392389773e-06, P174=3.7660074307345123e-06, P175=9.78753768971251e-06, P176=1.1420315980144172e-05, P177=7.713255066953326e-06, P178=3.3713341820541495e-06, P179=3.4605380746322363e-07),Row( P0=-782.8647830739137, P1=65.56876245515828, P2=9.702318192627356, P3=16.010325581393623, P4=-2.5936586297023414, P5=-5.09216844988709, P6=1.0140640754016386, P7=5.875087113665951, P8=3.6317680606808667, P9=-3.1735720255650963, P10=1.1610025681463647, P11=0.9157479579531979, P12=-3.515760626136504, P13=-1.8092034957974081, P14=-3.5380989218010406, P15=-0.7600305963496218, P16=-0.2231781830061873, P17=-2.6398127843011534, P18=-1.8060049777365508, P19=-4.351495471124669, P20=-3.363570445147544, P21=-2.4177753982923007, P22=-5.993270913675454, P23=0.5267122439233716, P24=-4.521397422593793, P25=-2.0549928612500814, P26=-4.542959902048565, P27=-1.833688203158481, P28=-2.9877196355760938, P29=-0.23451136121615726, P30=-2.7240705589133825, P31=0.4962433634120477, P32=-2.0993283645937053, P33=0.7583902473546481, P34=-0.9218028029318829, P35=0.25109030940204313, P36=1.3374540966098296, P37=2.522203226653019, P38=2.1916287500444236, P39=4.2119831944540955, P40=0.7313954263501512, P41=0.7234664445888868, P42=0.7190476667250292, P43=0.705241438045901, P44=0.6933654016034225, P45=0.7057910191741867, P46=0.7500009489771481, P47=0.7369262213340133, P48=0.7979711672766164, P49=0.8070727405577339, P50=0.7576652099933342, P51=0.7166398356873427, P52=6.234544538170832e-05, P53=8.567116035951687e-06, P54=2.7478345473060777e-06, P55=3.5152087205330744e-05, P56=0.0011430319130903777, P57=0.0075218352646494, P58=0.005404524060904547, P59=0.0020319216859514305, P60=0.0004121277962796841, P61=7.867322168290306e-05, P62=0.0001723239761621529, P63=0.00047672430286378265, P64=0.0006563932841643331, P65=0.001002714960974644, P66=0.000512891770523871, P67=0.00017606813151237066, P68=0.00010990644126134584, P69=0.0002806383261967842, P70=0.000493506027497675, P71=0.0003521736472894267, P72=0.0003530927866373668, P73=0.00027765076181932827, P74=0.00025578348277828543, P75=0.0002224380846225046, P76=0.0002162625986096207, P77=0.0006515071616461499, P78=0.002143795121401846, P79=0.00353351222752007, P80=0.0013648527542580431, P81=0.0002184165527191649, P82=0.00019475559726910477, P83=0.00019482598914947818, P84=0.0001736500892544113, P85=0.00013122664140635372, P86=0.00010733719309914066, P87=0.0001061135545781733, P88=9.926549114093641e-05, P89=5.994530737526403e-05, P90=4.4114936771793354e-05, P91=4.739880185174214e-05, P92=4.8105243226431117e-05, P93=2.8843859355616108e-05, P94=2.8007954452385153e-05, P95=3.117927717929269e-05, P96=2.272657442763709e-05, P97=2.2419026123537205e-05, P98=1.7832958097588965e-05, P99=1.3279252120500097e-05, P100=1.1943532156189685e-05, P101=1.1789277687494798e-05, P102=5.4908306514202295e-06, P103=5.918366652241824e-06, P104=1.1217449390585321e-05, P105=7.68741425814887e-06, P106=6.7798521210413305e-06, P107=7.839741978700373e-06, P108=3.907610907563517e-06, P109=3.4821554319767087e-06, P110=4.715810219195947e-06, P111=8.613578015806119e-06, P112=4.265756926170394e-06, P113=4.373943757545298e-06, P114=6.176651709285095e-06, P115=1.2201951988125133e-05, P116=1.0292788685022922e-05, P117=5.292767203001815e-06, P118=2.617666104498124e-06, P119=7.466362710908547e-06, P120=1.232274178710358e-05, P121=4.111545292172439e-06, P122=5.735476453766687e-06, P123=5.545060877154619e-06, P124=5.650094091166458e-06, P125=7.1866834945143605e-06, P126=6.480276779019648e-06, P127=9.820162034952075e-06, P128=9.73855203291766e-06, P129=1.0587651693232026e-05, P130=6.520763036432469e-06, P131=6.925679389640377e-06, P132=1.468231508973021e-05, P133=1.402331226135949e-05, P134=1.189547533793121e-05, P135=1.2261781885492129e-05, P136=1.342401596789578e-05, P137=8.374550647025218e-06, P138=3.2720506136927043e-06, P139=4.38645496970425e-06, P140=3.262470768349344e-06, P141=3.443081359129225e-06, P142=2.0427611404366893e-06, P143=1.6433701798223025e-06, P144=2.4035391151912914e-06, P145=1.042557964800754e-06, P146=3.3635569129654857e-06, P147=4.697609213089751e-06, P148=5.877408373383336e-06, P149=8.404703168480207e-06, P150=1.1499371985529776e-05, P151=1.0010634366701134e-05, P152=1.0913026753042917e-05, P153=1.577815031714282e-05, P154=1.0325952725393627e-05, P155=6.176640994328253e-06, P156=2.8905557533160583e-06, P157=3.605725724566417e-06, P158=6.336044875233096e-06, P159=4.294614487085118e-06, P160=1.3140411152055887e-06, P161=1.4888515564827282e-06, P162=1.506686192155633e-06, P163=3.3258499892475678e-06, P164=2.0917392967871537e-06, P165=1.8266724741837595e-06, P166=1.6845000224905073e-06, P167=1.0865611092402047e-06, P168=1.4928900148167667e-06, P169=6.855932449217112e-07, P170=1.1372690485098296e-06, P171=1.0929203103684965e-06, P172=7.809800584804914e-07, P173=8.767559526514699e-07, P174=1.2731063340655079e-06, P175=1.8695717065906282e-06, P176=2.109747726554372e-06, P177=1.9590278667174664e-06, P178=9.936005953333172e-07, P179=9.189881329242987e-08),Row( P0=-689.7111676833133, P1=70.06772472629542, P2=1.0534123722542124, P3=9.301417052587944, P4=-10.517120790310923, P5=-5.722395059896891, P6=2.5356543015024955, P7=1.3238718660591897, P8=-2.408605664699963, P9=-4.232228016378561, P10=-2.8606265381775016, P11=-0.5682196254797086, P12=-6.3548952978612325, P13=2.3987052989372897, P14=-6.732239968897784, P15=-1.7127010049206868, P16=-2.7138982161164895, P17=-0.42277216273470797, P18=-2.0768689654499863, P19=-4.231870384948528, P20=-1.586837073666962, P21=-6.083021000397686, P22=-4.774592518421876, P23=1.360437573068469, P24=-8.664959145290592, P25=-4.3741413282553, P26=-3.521291171374348, P27=-3.7710411505280566, P28=-4.803578281916404, P29=-0.030571448958678502, P30=-1.728544587953273, P31=5.108667228540258, P32=2.1267067400451536, P33=3.8180670029489088, P34=1.3701858641009967, P35=-0.08338456543694993, P36=-2.102915075461169, P37=0.9512993448539699, P38=0.8841047011507868, P39=2.542141505853299, P40=0.7652025770326856, P41=0.7198203170528399, P42=0.692866492886378, P43=0.7119688264812607, P44=0.7263142423044144, P45=0.6699051915332515, P46=0.6587025459532538, P47=0.6543445134090388, P48=0.6585531830158747, P49=0.6361144532364975, P50=0.6126104102999002, P51=0.6629271629770173, P52=3.730935776117812e-05, P53=1.2108838404528504e-05, P54=3.0412357179049974e-05, P55=0.0005123066844242155, P56=0.004053877778256272, P57=0.007662693150632672, P58=0.004575646066835209, P59=0.008554829974661915, P60=0.004338350201268211, P61=0.0006003948527361311, P62=0.0008698571949934974, P63=0.0011698139085986204, P64=0.0007419676780515732, P65=0.0022429948480886352, P66=0.005103625442175889, P67=0.01005006898715059, P68=0.008353107819351663, P69=0.0035673096482461935, P70=0.0015688550285758215, P71=0.0003991512903972087, P72=0.0002096867003236275, P73=0.0004075032910504642, P74=0.0010273378579629057, P75=0.0029717755005867685, P76=0.004009947348414709, P77=0.005235517716103545, P78=0.004254203095549937, P79=0.002117463690616551, P80=0.0008187786348807082, P81=0.0007265361574047011, P82=0.0013667153787146702, P83=0.012810352030792655, P84=0.02645087557900937, P85=0.013784772103027724, P86=0.0014531118834559778, P87=0.0002118381694594753, P88=0.00014620804312365902, P89=0.00040612700806321665, P90=0.006858033156335172, P91=0.014448645137636612, P92=0.00427711170795612, P93=0.00038599725718172233, P94=0.00010226481777369969, P95=0.00019538645660436584, P96=0.0007716378797544885, P97=0.0006507315004471641, P98=0.00017662140782712923, P99=5.3188220153938516e-05, P100=0.0003318843274624261, P101=0.0009162324731623289, P102=0.0002976527067337383, P103=6.180410330500965e-05, P104=0.00022484359196888246, P105=0.0013906210278187643, P106=0.0008444224947816206, P107=0.00015817098365840648, P108=0.000640243622257899, P109=0.0005831728735927015, P110=0.00021802874332571846, P111=0.00014348602918833276, P112=7.448852236282104e-05, P113=5.5582071744326874e-05, P114=3.0284524784119873e-05, P115=1.1386330983407949e-05, P116=2.074245833812522e-05, P117=2.9627909437123305e-05, P118=1.1509048337288735e-05, P119=1.9364336324170397e-05, P120=2.2247546475856905e-05, P121=1.3447689772989501e-05, P122=2.175628036030686e-05, P123=1.4830209027914444e-05, P124=1.6636364113797108e-05, P125=2.3068324309453265e-05, P126=2.5692953999585205e-05, P127=1.727269310406434e-05, P128=2.2612323964708446e-05, P129=7.095010625942338e-05, P130=5.950869432252117e-05, P131=4.185918465357657e-05, P132=3.982246017233583e-05, P133=1.9987101549163603e-05, P134=1.339148131466985e-05, P135=1.2173946403925867e-05, P136=1.0123852851674419e-05, P137=1.1932903319760476e-05, P138=1.916106173324935e-05, P139=2.2107070424240418e-05, P140=3.565381529401462e-05, P141=4.076826423310092e-05, P142=3.086214129555479e-05, P143=3.512555042273856e-05, P144=1.7283747645274514e-05, P145=1.5467864777540467e-05, P146=8.514263332997115e-06, P147=1.5662134797257663e-05, P148=2.4259965530507532e-05, P149=4.221183879786634e-05, P150=8.726525551621554e-05, P151=8.426616038164469e-05, P152=0.0001357120840730541, P153=0.00018024692168034574, P154=0.00017119596119689865, P155=0.00010862478231989617, P156=8.027316350821232e-05, P157=5.713170513946707e-05, P158=6.0049220097257355e-05, P159=4.187349901522558e-05, P160=3.675410070226896e-05, P161=1.628181286890089e-05, P162=1.5089704496796824e-05, P163=7.292578135887425e-06, P164=7.577947659684376e-06, P165=1.1837916687940225e-05, P166=1.4027113043692518e-05, P167=1.6242373104733718e-05, P168=1.8958078194701172e-05, P169=1.9336467826504548e-05, P170=3.015808686737196e-05, P171=2.2424375584840864e-05, P172=2.428253167811356e-05, P173=1.781816945837611e-05, P174=1.458565344793348e-05, P175=1.6506760596135815e-05, P176=1.834957122355343e-05, P177=1.8131934486931535e-05, P178=9.775253440500908e-06, P179=1.011270047959964e-06),Row( P0=-674.3071199459165, P1=76.28055675839067, P2=-6.288675626511283, P3=12.913469542694875, P4=-10.761580573942615, P5=-11.926145000506711, P6=-0.0061198201290205995, P7=-4.684834364876466, P8=-5.236640177301774, P9=-0.5312170165963128, P10=-3.65954919538806, P11=-4.460409512568672, P12=-4.966079559244874, P13=-4.598774124994073, P14=-1.9533920051194544, P15=-5.39645834045423, P16=-2.1956080315499698, P17=0.9739783764472327, P18=-4.8337725700456, P19=-1.2441285579238373, P20=-1.7055477112341813, P21=-3.054508367609707, P22=-3.9385094938206926, P23=-1.8984565713843395, P24=-4.835681339630806, P25=-2.828031870078208, P26=-3.7762365932182385, P27=-0.9828542389680845, P28=-3.483880489709337, P29=-1.9229038072037503, P30=-0.6015300318858978, P31=0.8565891245492893, P32=-0.9449261656247232, P33=1.955387107439445, P34=-1.18555968292031, P35=-0.6144384968262103, P36=-1.7627793621108612, P37=1.012757602336352, P38=-1.2053805986576374, P39=1.619464670009237, P40=0.7522508949774326, P41=0.7278253838986715, P42=0.7075123683156732, P43=0.6995425115279885, P44=0.7569906276258327, P45=0.7315897022130227, P46=0.7246677926709959, P47=0.7206578194153638, P48=0.7051305409341254, P49=0.7351577585620623, P50=0.7305013336948184, P51=0.7397398575418177, P52=1.830101197266511e-05, P53=4.2419711838167765e-06, P54=1.4256956272032516e-05, P55=8.886842309780349e-05, P56=0.0004723012254351684, P57=0.0010833263546837807, P58=0.0011953211698043436, P59=0.002114118148416636, P60=0.0014508858232299851, P61=0.0006487813791073774, P62=0.001047414683346079, P63=0.001180564467512797, P64=0.0008990201073002799, P65=0.001648641354064027, P66=0.003118768022539604, P67=0.006454014559659077, P68=0.005383345589115406, P69=0.0038740601947054175, P70=0.0015730278817820233, P71=0.0010381213087170785, P72=0.002137273983243961, P73=0.002232358168095795, P74=0.0021903634307899942, P75=0.002997702254213587, P76=0.007651448842818674, P77=0.02606162747674572, P78=0.017540514408282908, P79=0.006642105895119403, P80=0.0024892067468720723, P81=0.0012567653828784839, P82=0.0019112548063500464, P83=0.0033510053526523064, P84=0.0031228870490594886, P85=0.003359611595296918, P86=0.0013249266850831035, P87=0.0005047801323475495, P88=0.0008184129577262875, P89=0.010126979052763097, P90=0.02545040227365079, P91=0.005175882187864859, P92=0.0006577960608605374, P93=0.00028665657713601, P94=0.0003591383349706688, P95=0.002252500498140721, P96=0.002353050408754797, P97=0.0005117163746259952, P98=0.0003019934292823448, P99=0.0002799947953507611, P100=0.0011961353076413229, P101=0.0007400414537399237, P102=0.0001647389721573913, P103=0.00020780293915798498, P104=0.0013854426039744902, P105=0.0004976951878398113, P106=0.00015005011531980044, P107=0.0008021131953944633, P108=0.0017497167928202035, P109=0.00031345197796387245, P110=0.0004163932924785621, P111=0.0030503432411261564, P112=0.0014626590409134717, P113=0.00015376653661866593, P114=0.0001516970591713834, P115=4.018954625989904e-05, P116=9.304621871990418e-05, P117=7.905603449640394e-05, P118=3.931312308104158e-05, P119=9.886598344117014e-05, P120=8.548882910066485e-05, P121=8.880626906429704e-05, P122=7.938711350048128e-05, P123=5.138615244176159e-05, P124=7.164795596562961e-05, P125=8.370037921976804e-05, P126=7.363167955733704e-05, P127=0.00028302637396032134, P128=0.00041897402196014184, P129=0.0003957347405379469, P130=0.0007157566663912987, P131=0.0002321868718685295, P132=0.00020333317772097914, P133=0.00014674396023851532, P134=0.0001760020699393719, P135=0.0002462872790740269, P136=0.00023318405970664162, P137=0.00038800272778052826, P138=0.0003108240870375443, P139=0.000321357871590419, P140=0.00022883944299036514, P141=0.0002014449701377004, P142=0.0001899936554314884, P143=0.00010912026575158003, P144=8.910706514980401e-05, P145=0.00011021453224702328, P146=0.00010082409567386871, P147=4.9940744750993914e-05, P148=3.675088405147868e-05, P149=1.9114800107263538e-05, P150=1.0841056948530007e-05, P151=5.931266580421188e-06, P152=5.760282606053983e-06, P153=6.855651026604782e-06, P154=1.1268863480017307e-05, P155=1.1051804431247746e-05, P156=1.5163818481910125e-05, P157=2.4337363747484198e-05, P158=3.567391038751581e-05, P159=3.038779853553359e-05, P160=1.777004242139295e-05, P161=1.764940929056857e-05, P162=1.1407844826402206e-05, P163=7.1044476015896886e-06, P164=4.396399058490247e-06, P165=3.282130070852798e-06, P166=2.955594038761441e-06, P167=2.931010005908422e-06, P168=2.8534102628537667e-06, P169=5.502211462933432e-06, P170=6.615364946963777e-06, P171=8.166215687537876e-06, P172=8.434136178637222e-06, P173=4.45933682284665e-06, P174=4.060707251846525e-06, P175=6.373242240219443e-06, P176=6.372174092691402e-06, P177=3.074374254742984e-06, P178=1.614600068731536e-06, P179=2.8555373780768325e-07),Row( P0=-748.4013103622004, P1=78.62962997838649, P2=-13.617800012991067, P3=12.71636850137863, P4=-11.287789301024928, P5=-14.643720872594796, P6=0.5696836904804089, P7=1.7526980112124573, P8=-3.1423334117343704, P9=-7.8340179236371315, P10=-3.6954332077532563, P11=-3.558663745789782, P12=-6.932077815686508, P13=-4.421247697703093, P14=-2.589642783825801, P15=-1.4695906653694024, P16=-1.8056760241203993, P17=0.15658355175966468, P18=-1.831410014096817, P19=-3.328312282988092, P20=0.10250845345233911, P21=-2.8662773814300224, P22=-2.801222513993456, P23=-0.7239366010213257, P24=-5.928768419217776, P25=-0.646125982297977, P26=-3.920818936423616, P27=0.27140409563714313, P28=-2.8723440660259967, P29=-0.8564670838179371, P30=-2.0628039459985543, P31=0.877281342015523, P32=-3.7141289202831462, P33=1.8273090929743208, P34=-0.5377028826554707, P35=-0.4672162491730604, P36=-0.4153267976379559, P37=1.2257484890497607, P38=0.6880723410644346, P39=1.521868242320508, P40=0.7592460769104802, P41=0.734766465153614, P42=0.7763438803347562, P43=0.765997101260857, P44=0.7398644860307373, P45=0.7105523604210545, P46=0.709073214599113, P47=0.687825696509015, P48=0.7412849094119194, P49=0.7792781868072585, P50=0.7721739013009488, P51=0.7348496379005586, P52=6.802849354293156e-06, P53=1.9106800888372094e-06, P54=1.4526108495390839e-06, P55=2.349298852770481e-05, P56=0.00027420681633073905, P57=0.000992221157722129, P58=0.0006713490623100226, P59=0.0004942654969557658, P60=0.00047757906685896753, P61=0.00032983236988845337, P62=0.0009238862819293665, P63=0.0014604667459188567, P64=0.0007445906516756142, P65=0.0010188503009434633, P66=0.00182096575809447, P67=0.002045803614421109, P68=0.0014117970114535115, P69=0.0008554891876823979, P70=0.0006731662708264779, P71=0.00045671591667387467, P72=0.0002599685648274971, P73=0.000515859493682358, P74=0.0008113372140808019, P75=0.0014555430910844517, P76=0.0020804788779611593, P77=0.003018112917254875, P78=0.0028698768338870314, P79=0.002056254645656147, P80=0.0012660811886614299, P81=0.0008235902804949394, P82=0.0004495929483264677, P83=0.0002545105380615259, P84=0.0004924561513510983, P85=0.0006906034324858565, P86=0.00046539866740173024, P87=0.0025522986786366457, P88=0.0033876214412549392, P89=0.0006763667773312972, P90=0.00015814828667682482, P91=0.00015136758121338077, P92=0.0006060379978031721, P93=0.002584625058687214, P94=0.0011525585690318647, P95=0.00012518717076101413, P96=4.3772676478048864e-05, P97=0.00034128046239955095, P98=0.0007011912629863949, P99=0.00015495862820479067, P100=2.080998029579497e-05, P101=6.063719666117043e-05, P102=0.00021546689791386946, P103=8.350123411338471e-05, P104=1.2913641383368393e-05, P105=0.00021759647835963193, P106=0.0005351343828065188, P107=8.721762489975324e-05, P108=0.00014141577094417725, P109=0.0004986083033579736, P110=5.044792297628118e-05, P111=0.00012125584176584041, P112=0.00020777119605630928, P113=3.8476468667459736e-05, P114=2.2769604468205868e-05, P115=1.1659253862139648e-05, P116=2.3265139440655994e-05, P117=5.4513200445732994e-05, P118=4.0567682892810476e-05, P119=2.2129526903989587e-05, P120=2.6297390848186555e-05, P121=3.546023967033541e-05, P122=1.9556157767174044e-05, P123=3.9909722100409686e-05, P124=5.012049326271966e-05, P125=0.0001010756802210162, P126=7.496156813700642e-05, P127=5.635031889541028e-05, P128=5.265669459282783e-05, P129=0.00012932438582445883, P130=4.4606887572005726e-05, P131=3.060635750199514e-05, P132=2.2284524000827892e-05, P133=2.5740883884630974e-05, P134=2.4171800491069755e-05, P135=2.3393043614798907e-05, P136=2.9792538235772516e-05, P137=4.115664566614764e-05, P138=4.34548680564928e-05, P139=1.597922847742946e-05, P140=1.4894846587977711e-05, P141=2.0964855599378752e-05, P142=3.348762606655283e-05, P143=2.704257921687265e-05, P144=2.178378308355811e-05, P145=9.454363612122535e-06, P146=8.24364115918948e-06, P147=6.241706342190021e-06, P148=3.067125802403962e-06, P149=3.0634076174074438e-06, P150=6.85850919278096e-06, P151=6.748639865695495e-06, P152=7.03472779716008e-06, P153=8.11165313837633e-06, P154=3.3614343743154017e-06, P155=3.628983728935116e-06, P156=2.6619593602820586e-06, P157=2.3873970348864118e-06, P158=2.7663211835364073e-06, P159=4.176370347276987e-06, P160=1.9202052897525086e-06, P161=9.7878577785101e-07, P162=6.340464772620777e-07, P163=5.710906177634277e-07, P164=8.754697710335487e-07, P165=1.4749497690506482e-06, P166=1.6767014136241107e-06, P167=1.6234217922293152e-06, P168=9.519160065880375e-07, P169=8.224784928491951e-07, P170=6.398251571294709e-07, P171=2.4440255653918644e-07, P172=5.176934346514609e-07, P173=7.651071599941004e-07, P174=1.312136447370056e-06, P175=1.4048178262290764e-06, P176=1.2544194346500702e-06, P177=1.8457159183107132e-06, P178=7.459991567919006e-07, P179=6.399779666129612e-08),Row( P0=-523.2578766310103, P1=55.103011758078694, P2=-19.773464124177195, P3=-0.422864765612208, P4=-12.436787050381371, P5=-22.048566732592086, P6=-7.486601872623895, P7=0.353430354466527, P8=-0.029466747698099795, P9=-2.6373787295497944, P10=-12.576410407229439, P11=-3.3731671652231525, P12=-7.686747830028429, P13=-5.971936136426986, P14=-2.0062477450375105, P15=-0.5632449561199941, P16=0.606375490567035, P17=12.21400582322071, P18=6.613487352731761, P19=14.328522497357657, P20=5.980451702871307, P21=0.9535999987218025, P22=2.8354800780686613, P23=2.8072900813294113, P24=0.5876793721835362, P25=3.5680610714025245, P26=3.276930039867265, P27=1.5008925505879949, P28=-1.0347890714116244, P29=2.013163585132894, P30=1.7236316479497105, P31=3.205354811571003, P32=1.8702340151902246, P33=3.2839081793191665, P34=0.4920737499340694, P35=2.1639825479631853, P36=3.766946362744603, P37=2.6808726412799255, P38=2.7284710833455885, P39=3.055475382847336, P40=0.6188438242361286, P41=0.572618182742553, P42=0.540183169912248, P43=0.5457209101941358, P44=0.5617066763999555, P45=0.5724717510497623, P46=0.5564105040349467, P47=0.556072063068395, P48=0.5944617970415768, P49=0.6866430538802091, P50=0.724346051405637, P51=0.6580086871629255, P52=0.0011734807332090536, P53=0.00021300806025570737, P54=1.979788736565496e-05, P55=1.222441950248534e-05, P56=7.660426687393631e-05, P57=0.00046601552215867715, P58=0.0007934830215547365, P59=0.0016721675253427318, P60=0.005638864781595496, P61=0.013507555370047981, P62=0.04903856362665443, P63=0.08935536372074127, P64=0.10105503646362389, P65=0.2682439731635687, P66=0.25710481419818493, P67=0.1167486803338553, P68=0.030258992688725495, P69=0.006268273876390944, P70=0.008197433411373993, P71=0.0045850337054590224, P72=0.0035875448125358774, P73=0.00993015504177924, P74=0.03066093584926897, P75=0.03699124181230866, P76=0.033312180903126626, P77=0.10599220621678472, P78=0.358303952412691, P79=0.8929291635781632, P80=0.9384680567510084, P81=0.5223685445048009, P82=0.8557527346633694, P83=0.972511925284185, P84=0.5463847435820129, P85=0.12122142373142851, P86=0.0668835133832836, P87=0.1639886120450255, P88=0.2075837366601437, P89=0.2007388560854088, P90=0.6803766205327654, P91=2.1102387330559114, P92=1.014851725569696, P93=0.41208986926326174, P94=0.4634818184512581, P95=0.3481991173912144, P96=0.26886995934000435, P97=0.12499068503972705, P98=0.015774919647035614, P99=0.013147736085479452, P100=0.015664397978848197, P101=0.004924300430029443, P102=0.005804136948385857, P103=0.013894356188659638, P104=0.030881013752611004, P105=0.050057657018147954, P106=0.0596892236204057, P107=0.054173441415864054, P108=0.025474314284084826, P109=0.02928092347423519, P110=0.03327856623042506, P111=0.011365523205927215, P112=0.06295114428180787, P113=0.049407911281074725, P114=0.008497383894431044, P115=0.010015715346048603, P116=0.0057054387698396245, P117=0.020372605523836716, P118=0.027700882851373218, P119=0.007982570977539793, P120=0.0030583032761569867, P121=0.006203266291732086, P122=0.005630158893700041, P123=0.00855559442722818, P124=0.0034606856059909473, P125=0.002572407778367923, P126=0.004723952698214051, P127=0.002643909603136264, P128=0.0038547450616329606, P129=0.00495303400953207, P130=0.009595777000883925, P131=0.02504134596103615, P132=0.008555497970692955, P133=0.002309735275525253, P134=0.0015374042987455797, P135=0.0009845848055216534, P136=0.0016925128441928933, P137=0.002140902324874241, P138=0.005736773467811932, P139=0.004364785861094932, P140=0.0029078686893691274, P141=0.0017749911478918293, P142=0.0026373081728885504, P143=0.0019349202457021471, P144=0.0020094096366232678, P145=0.0011426691390604103, P146=0.00048388018737956175, P147=0.0002679923065516309, P148=0.00021884088862142824, P149=9.280837991168212e-05, P150=0.00012278434150484968, P151=0.00022180626545830652, P152=0.00028633412619098503, P153=0.00032106528613044734, P154=0.0005225469163761741, P155=0.0005065305203492302, P156=0.0005658940677777237, P157=0.0006380595537881753, P158=0.0003928301379747376, P159=0.00025856730409174735, P160=0.00020042335743710704, P161=0.0002686232386127435, P162=0.000298884484008013, P163=0.0002793828088976358, P164=0.0004108267736007022, P165=0.00021863911226435695, P166=0.00013104938060934157, P167=0.00020366666231375573, P168=0.00014140263527344564, P169=0.00010357433166921195, P170=0.00016354425159676125, P171=0.00015039460490155624, P172=0.00014470054526723517, P173=0.0001300644651753876, P174=9.119465889332256e-05, P175=8.004056770839939e-05, P176=6.796960656613337e-05, P177=2.6816793420648974e-05, P178=8.994531315823747e-06, P179=8.061244063189072e-07),Row( P0=-681.0939977838541, P1=81.44503430733297, P2=-5.163551359840306, P3=12.819084134065811, P4=-1.8752964485879111, P5=-13.47880845123912, P6=1.5751395078429582, P7=4.2249204252832415, P8=0.808184337764122, P9=-1.5991198558481887, P10=-2.155569293776357, P11=1.943445380114833, P12=-4.618453247933238, P13=-3.0444521449984183, P14=-1.0713974063912466, P15=0.7571980125013672, P16=-2.743185728760652, P17=0.6954635682923946, P18=-2.573392736005005, P19=-2.471229549562142, P20=-2.1107831844951432, P21=-1.384206293690726, P22=-4.441231957042396, P23=2.988714186777493, P24=-2.748886565694207, P25=0.33498844999401084, P26=0.17979762320674614, P27=2.2700353609241373, P28=-0.0686098848465382, P29=2.298067063971183, P30=1.2031333670698372, P31=4.035577791631254, P32=2.534974978608514, P33=4.518437294516257, P34=4.146180589996236, P35=3.3221635830271072, P36=2.266120137967381, P37=1.96282943729316, P38=0.2630287698090695, P39=2.1597145178171155, P40=0.7762284090936951, P41=0.727110963620236, P42=0.6872174150033051, P43=0.6978621266435003, P44=0.6897791817610007, P45=0.6915933859570188, P46=0.7114941005036886, P47=0.7095716699765774, P48=0.7143695339882246, P49=0.7148216304878131, P50=0.7278204081158337, P51=0.7596744071303757, P52=0.0005354548652093872, P53=6.446740307189124e-05, P54=9.97247777211448e-06, P55=2.219404439217836e-05, P56=0.00034835742049235316, P57=0.002318568156972777, P58=0.006737203569702837, P59=0.010948902695048352, P60=0.008351333552495422, P61=0.010007147180004092, P62=0.00757949319190364, P63=0.0015499408525055246, P64=0.0016658028935205374, P65=0.007056596887855433, P66=0.018526711029940895, P67=0.021730949313778666, P68=0.011859840988541035, P69=0.007146162947702852, P70=0.005545786143308753, P71=0.008204760339740517, P72=0.024309792814448172, P73=0.023106514797855526, P74=0.008894191147324275, P75=0.007178060135698033, P76=0.00843143360623179, P77=0.014287936249201333, P78=0.01006767115689262, P79=0.008490183563245286, P80=0.011421693171937522, P81=0.014671477076801623, P82=0.01647331007691725, P83=0.013643528786988305, P84=0.023131549081781187, P85=0.01481521076061028, P86=0.006230022533114937, P87=0.007650978564351664, P88=0.013793261871386391, P89=0.014900659473078772, P90=0.010020842212139822, P91=0.0044825208012842736, P92=0.0015147044788129873, P93=0.0013137622054609675, P94=0.002245816628210029, P95=0.010281583213781347, P96=0.009014282820936012, P97=0.0035168194787676167, P98=0.0011976019328677134, P99=0.0005695784885037227, P100=0.0007409953717467478, P101=0.0013479344503596429, P102=0.0009326287558495313, P103=0.000764082540742133, P104=0.00048723421014074743, P105=0.0014671096746342125, P106=0.002066810423792787, P107=0.0008035813788213159, P108=0.00015195913317617404, P109=0.00041314917569339193, P110=0.0017114403735915002, P111=0.001096217394255386, P112=0.0002663582514152658, P113=0.00020590810473934698, P114=0.00022379931748325096, P115=0.0004062310010637675, P116=0.0005014886474829746, P117=0.0002793965390555339, P118=0.00033682787009452724, P119=0.0008088092320504953, P120=0.000584884832691, P121=0.00043960173878964575, P122=0.00027046937134395083, P123=0.0009232457583852947, P124=0.0006136388646661381, P125=0.0003615857842302937, P126=0.0004955316369174476, P127=0.0003812045120020966, P128=0.0005461264767078375, P129=0.0006783372159605907, P130=0.0003510819844164867, P131=0.00033243916771299857, P132=0.00017093209755029602, P133=0.00011598098753735867, P134=0.00011333596559927493, P135=8.284163403980104e-05, P136=7.344853168429145e-05, P137=8.630988710721846e-05, P138=8.81395363361693e-05, P139=0.00011012959822249407, P140=0.00010755256638431459, P141=0.00011236029714865944, P142=0.00013691397837014142, P143=6.68772622433223e-05, P144=3.3598221827079125e-05, P145=1.5051439427334486e-05, P146=5.224595180316422e-06, P147=6.967414185318055e-06, P148=9.894528939033754e-06, P149=9.752364065320463e-06, P150=1.949725897217843e-05, P151=1.98446244161737e-05, P152=1.9678729061807816e-05, P153=2.1620753505530374e-05, P154=1.6282464931171383e-05, P155=2.4515666035084027e-05, P156=2.549962145718795e-05, P157=1.3406724934094384e-05, P158=2.3166902188763595e-05, P159=1.8718164001900313e-05, P160=1.179026060820911e-05, P161=1.1123157328611566e-05, P162=1.2485203790644885e-05, P163=1.0765622500106944e-05, P164=9.711055525909716e-06, P165=7.393850673078314e-06, P166=5.601116163973851e-06, P167=5.561992881237914e-06, P168=5.368894619361499e-06, P169=5.54183608120723e-06, P170=3.183013680020183e-06, P171=3.5566984485431832e-06, P172=5.62837072608707e-06, P173=4.486455348796903e-06, P174=4.508253704409442e-06, P175=6.34079104188955e-06, P176=7.377648487783973e-06, P177=9.101138433383736e-06, P178=1.9888887922260296e-06, P179=2.0436338131311538e-07),Row( P0=-740.8893628126445, P1=85.23326686303034, P2=-13.778110099890712, P3=11.84712878960774, P4=-9.395942045951786, P5=-12.871187696501996, P6=-1.6425943664127516, P7=3.4828227059376515, P8=-1.486825787023713, P9=-6.565832239861154, P10=-1.1489364709199579, P11=-4.20860462556477, P12=-6.8421036522425025, P13=-4.32402066940038, P14=-3.019003866621096, P15=-1.110120879048193, P16=-2.3040877558483897, P17=0.08716014432343651, P18=-0.4704787388824726, P19=-4.077295194729358, P20=0.03838360123152203, P21=-1.2901223741033192, P22=-4.6137406316243466, P23=0.2665388270458489, P24=-5.698948657682516, P25=-3.2360335508519764, P26=-4.387511445104927, P27=-0.008547952352959654, P28=-2.532336454596031, P29=-2.2623415941475122, P30=-1.2875007843704525, P31=-0.2801937993699087, P32=-3.154814960800188, P33=0.4528046011536063, P34=-1.4630864017346088, P35=-1.8502413148471122, P36=-0.5199338976591441, P37=0.7287257965616861, P38=-1.4510296619520389, P39=1.386469627765758, P40=0.7367350392485353, P41=0.7779187337405136, P42=0.7836975060761963, P43=0.7670318458169927, P44=0.7291092495193505, P45=0.6774100316284567, P46=0.664866793288137, P47=0.6906267426927304, P48=0.7554391858862219, P49=0.7853373538110123, P50=0.7981255390730976, P51=0.7801476816588697, P52=2.4961630119497362e-06, P53=1.0282808401211235e-06, P54=3.765952032662753e-06, P55=9.244773183492803e-05, P56=0.00039669786288054127, P57=0.0005463091778895656, P58=0.0003742667547705858, P59=0.0003622864204975105, P60=0.0005910349610608906, P61=0.000731843877237881, P62=0.000731966978686741, P63=0.0005905896461890182, P64=0.00043965177624013667, P65=0.0009885231609225613, P66=0.0014930772281455099, P67=0.0010842127357350217, P68=0.0010159157141082116, P69=0.0010186135305582662, P70=0.000822391269809267, P71=0.0006605771854845929, P72=0.000621608368492367, P73=0.00045689057847377176, P74=0.0004301177504590589, P75=0.0007581508535547771, P76=0.0015830017171759871, P77=0.004913253180050836, P78=0.005336589675563243, P79=0.0033964093757071438, P80=0.0016110086448708767, P81=0.0013076049168569434, P82=0.0010783641571392012, P83=0.000577553767310059, P84=0.0006371556583236488, P85=0.0014096753062032205, P86=0.001485067797397249, P87=0.0011964925741086993, P88=0.002152362366164026, P89=0.0008589520561929525, P90=0.0006302262463340289, P91=0.00040622959207844517, P92=0.0007485158536845396, P93=0.0011140334459481677, P94=0.0002242573547722845, P95=8.71888737671196e-05, P96=8.428143324293334e-05, P97=0.00031573132275878116, P98=0.00014922034353494527, P99=7.839519579732436e-05, P100=8.443649991427801e-05, P101=0.00012788894920334008, P102=4.3175573730006984e-05, P103=5.085086951861475e-05, P104=8.087411467125953e-05, P105=4.3141666568644796e-05, P106=8.424272685038233e-05, P107=0.00018819518054243956, P108=9.178471401905252e-05, P109=4.27587013346913e-05, P110=6.117389562814265e-05, P111=6.359480390086114e-05, P112=4.213119364853725e-05, P113=1.8542448653840008e-05, P114=1.4319007684191533e-05, P115=2.1294932885408028e-05, P116=6.0164383835213446e-05, P117=7.944231858680229e-05, P118=2.641132136891104e-05, P119=0.00015605688352015288, P120=0.00010763986316370648, P121=3.7110277071378795e-05, P122=4.5796684484120906e-05, P123=3.4640848087497155e-05, P124=4.845590854986127e-05, P125=5.0563674089375936e-05, P126=4.690188268621322e-05, P127=6.28496974496391e-05, P128=8.840781553153137e-05, P129=0.0001040151299078084, P130=9.96236232785716e-05, P131=3.0185838273604363e-05, P132=2.104600035084453e-05, P133=2.265331374146692e-05, P134=1.7841319314877248e-05, P135=1.3986751984945356e-05, P136=1.2174315738049345e-05, P137=1.1463312923499198e-05, P138=1.0409014733182478e-05, P139=9.762333661457339e-06, P140=1.1711419915346381e-05, P141=1.8253296193346078e-05, P142=1.5229585955329414e-05, P143=1.0803512477812426e-05, P144=5.625060861573943e-06, P145=3.8483566597183245e-06, P146=2.086621135952133e-06, P147=1.7370497854724645e-06, P148=2.836671114794169e-06, P149=4.884626407166417e-06, P150=1.5119343467571994e-05, P151=1.6252688710380768e-05, P152=1.1533427671196418e-05, P153=7.676703015818104e-06, P154=4.455446524606332e-06, P155=3.0598098459809636e-06, P156=2.610992144049082e-06, P157=2.7590917403402803e-06, P158=1.8616857882457706e-06, P159=9.047404196409013e-07, P160=1.7390540106277472e-06, P161=3.1893572198634898e-06, P162=1.5948068343820224e-06, P163=1.0544811957983073e-06, P164=1.4424670920047764e-06, P165=1.1305818083284956e-06, P166=7.8993206989795e-07, P167=6.475296521286454e-07, P168=3.5057088356494215e-07, P169=2.944034261304255e-07, P170=2.7450525068848544e-07, P171=3.9475200428765125e-07, P172=5.644264792475732e-07, P173=4.7364146955751865e-07, P174=7.653703338830837e-07, P175=7.052981621050673e-07, P176=1.0248063005647244e-06, P177=1.307198905470265e-06, P178=4.3320157165436823e-07, P179=5.083079660825975e-08),Row( P0=-509.55948011152196, P1=49.13107271136289, P2=-31.682864171627482, P3=-3.0067319022562398, P4=-15.375861822272983, P5=-23.5424469590921, P6=-1.717877923703406, P7=1.083804661886595, P8=-5.239290241147652, P9=-2.651861985501822, P10=-13.204473744578726, P11=-4.150431178924533, P12=-5.3585674869766455, P13=-5.344594220871615, P14=2.607079511789501, P15=11.849140825006735, P16=9.446845875917438, P17=17.099480472096133, P18=5.523685433792454, P19=6.8951865370396845, P20=1.5518038328231287, P21=1.3208954172722303, P22=6.294143923648039, P23=6.81972050953917, P24=1.2695101699237041, P25=-1.4289280421127932, P26=-1.306488866161504, P27=0.9624929948004962, P28=2.9836998482595387, P29=6.217528474370217, P30=0.9667687856926604, P31=3.622295879482228, P32=0.8093623778335397, P33=3.441246762907486, P34=0.42061196614389806, P35=-0.7631873666362361, P36=0.21005582772744438, P37=0.3072536516350984, P38=0.8588469083254419, P39=2.4358769475634965, P40=0.5827742172269343, P41=0.5852713356043581, P42=0.5343108770518739, P43=0.48229155730054146, P44=0.4797958018901304, P45=0.5339532061181903, P46=0.6109774994647098, P47=0.6170358452414293, P48=0.6316890508664125, P49=0.6405176680958602, P50=0.6179468378523011, P51=0.6151257093060519, P52=0.0015341709448166926, P53=0.00021876559633184528, P54=4.8133705202366156e-05, P55=3.312704273160402e-05, P56=3.992889824982629e-05, P57=0.00016470166229966464, P58=0.0003607614837479754, P59=0.001146460175220181, P60=0.0035166321091261235, P61=0.007067932257031292, P62=0.015137206985687386, P63=0.02617154270487513, P64=0.04080221310288747, P65=0.09782188370367967, P66=0.17833591478653238, P67=0.2534380487005598, P68=0.18249910408571923, P69=0.10929816439514066, P70=0.04412463322670109, P71=0.007287985399628019, P72=0.003188495529245789, P73=0.0037719778090585885, P74=0.005626392149128973, P75=0.008255515815347973, P76=0.012236993720799477, P77=0.045065417325600786, P78=0.09351882476630029, P79=0.14349070549007178, P80=0.21616763368977093, P81=0.4389368041852019, P82=0.7802339355521907, P83=1.2982256099889204, P84=1.379172405729544, P85=0.9812204736498555, P86=1.7200084254917354, P87=3.218017037894113, P88=2.050544369850584, P89=0.1552285934836906, P90=0.07933673233032147, P91=0.09917532447964343, P92=0.09629998204754199, P93=0.076927821952218, P94=0.12959567384895274, P95=0.3663473407353171, P96=0.7673110570814163, P97=1.1456815392791884, P98=1.5217192944804794, P99=0.8541472892573576, P100=0.1973353175992191, P101=0.011147644112486473, P102=0.0018736188893301506, P103=0.009595697830947316, P104=0.022862551709887876, P105=0.021152479489913216, P106=0.02125297685397832, P107=0.02246919513924616, P108=0.02990845803450525, P109=0.014012017561737615, P110=0.0083513329008651, P111=0.019164445748527832, P112=0.020959048348743622, P113=0.009408447215685397, P114=0.039939385839501435, P115=0.037366687083608185, P116=0.03637848956252209, P117=0.05956513354205826, P118=0.029986811980655233, P119=0.01812173754990886, P120=0.013966425237538896, P121=0.012684810457890822, P122=0.02149335438707522, P123=0.01404973486612874, P124=0.018092133357960786, P125=0.013665295666000376, P126=0.01615938237518822, P127=0.006273138908567296, P128=0.013789342480652644, P129=0.010459383242144297, P130=0.00973146606145161, P131=0.016170374539289923, P132=0.025948522667192536, P133=0.01094960537822401, P134=0.007594349461804333, P135=0.0052422451260425796, P136=0.005384786433384005, P137=0.010707958166330864, P138=0.00840414879140985, P139=0.005445053607725734, P140=0.005007948449665159, P141=0.005418533125851046, P142=0.0029032826502329127, P143=0.0025477770184162405, P144=0.0011348664027330513, P145=0.0005034173744481497, P146=0.00022976204981945688, P147=0.00023760359612684919, P148=0.0002009895895015852, P149=0.00012100658226955844, P150=0.00023042903265302906, P151=0.00019167495967207382, P152=0.0002213741681192024, P153=0.00029355855839343666, P154=0.00022010941483080032, P155=0.00016588898537831263, P156=0.00041664839714679547, P157=0.00042941195736251336, P158=0.00033732545577724616, P159=0.0003006019472469481, P160=0.00025676027756256937, P161=0.000250363172352292, P162=0.00022874564011594095, P163=0.0004516413449085398, P164=0.0004815357082408587, P165=0.0002729666860904378, P166=0.00023448340345431614, P167=0.00022892170294085972, P168=0.00020806359886577802, P169=0.00021265138792010392, P170=0.0002351353319150776, P171=0.00024865552819873353, P172=0.00021076910557563076, P173=0.00017198256104987002, P174=0.00017319026939199077, P175=0.00010729035318466725, P176=0.00010270538831128741, P177=5.048420233267907e-05, P178=1.4524464191658341e-05, P179=1.6570303205214327e-06),Row( P0=-730.4275391545254, P1=70.6012628675769, P2=0.658491649796975, P3=20.404391783287743, P4=-10.176195667209761, P5=-9.428595749008341, P6=2.7736332711641953, P7=6.499513124962679, P8=-2.119573029956612, P9=-6.5179531532822566, P10=2.167908747261959, P11=1.1308768795698279, P12=-8.443173692206416, P13=2.002263552891382, P14=-5.706882261414512, P15=-2.680838044656433, P16=0.1790057292977936, P17=-2.2619290022829444, P18=-0.5823267378385805, P19=-4.021482486026854, P20=-1.1739444416481568, P21=-1.5841952944426447, P22=-6.157924972869653, P23=3.937544432341331, P24=-8.27950771064015, P25=-3.7312405834138787, P26=-3.539789523433294, P27=-1.7107064856704406, P28=-5.526444829137871, P29=-1.304245313381677, P30=-4.20446487216364, P31=1.6010191080266567, P32=-1.6490585311344053, P33=0.399250793245547, P34=1.0003068582196204, P35=0.6202392054199792, P36=0.28915578476348003, P37=2.5821368273865293, P38=1.6224166614325786, P39=2.852461915759306, P40=0.6982998618345214, P41=0.7408305229870696, P42=0.7476961457276098, P43=0.692847489377759, P44=0.698711470908907, P45=0.6659993394837391, P46=0.6463444003635482, P47=0.6736219822230809, P48=0.6647626594911925, P49=0.701856846048809, P50=0.6905866018672115, P51=0.6642680835341259, P52=7.84798403852566e-06, P53=3.153886068929414e-06, P54=9.407014615319758e-06, P55=0.00016161230309492716, P56=0.0023626801408775503, P57=0.00772510493798247, P58=0.005847089252410523, P59=0.004794515057067619, P60=0.0013041354385021605, P61=0.00022845169418836013, P62=0.0008049507017844021, P63=0.0013448977701267626, P64=0.0014871946792570526, P65=0.0036559485642107753, P66=0.006294174100335439, P67=0.007265893469478282, P68=0.00288797353577446, P69=0.0005394929323157805, P70=0.00025405508089709707, P71=0.00031320470913261307, P72=0.0007691103698939034, P73=0.0014348662974036563, P74=0.003082850604898253, P75=0.0064382789535343595, P76=0.005378430330726516, P77=0.0023658033873328586, P78=0.001058852693171928, P79=0.0014136879010541157, P80=0.0017426973274282363, P81=0.0013747765145988538, P82=0.0007961806367414188, P83=0.0016192623795270333, P84=0.0018142546351112927, P85=0.0005623233793475174, P86=0.00030543181166010365, P87=0.00019709191820750302, P88=0.00011332948133285127, P89=9.341906024953119e-05, P90=0.00032572170611294514, P91=0.0004889686931079744, P92=0.00029585199056755715, P93=7.839595010087411e-05, P94=6.110334112739479e-05, P95=7.952357035912101e-05, P96=6.656991514152151e-05, P97=4.122463191333551e-05, P98=4.139777282223038e-05, P99=6.912764932296714e-05, P100=5.737738478661028e-05, P101=3.212994175112348e-05, P102=1.982206532351054e-05, P103=1.9389165256516405e-05, P104=1.7099663653556796e-05, P105=1.4842968992826829e-05, P106=2.8244101315262585e-05, P107=6.132532362945333e-05, P108=1.8460049599918027e-05, P109=1.398576861952093e-05, P110=2.8611215724762182e-05, P111=2.541808868840435e-05, P112=2.7599806213677945e-05, P113=8.93605106353076e-06, P114=5.200666336071028e-06, P115=5.605292673021245e-06, P116=1.8909920390859845e-05, P117=2.1750565174128086e-05, P118=7.396394203400845e-06, P119=0.00012017909255526478, P120=9.567400975804075e-05, P121=3.483407040359559e-05, P122=8.80234300405397e-05, P123=2.5759850552496732e-05, P124=3.974554579863832e-05, P125=3.4955851183946724e-05, P126=4.425777693845313e-05, P127=3.4133349055603466e-05, P128=8.073331137042735e-05, P129=9.8550516555358e-05, P130=5.554190656531685e-05, P131=2.557571807681021e-05, P132=2.4369514153496078e-05, P133=1.2385724240076865e-05, P134=1.0781777133028316e-05, P135=8.46596768016312e-06, P136=5.814700143361883e-06, P137=6.408839962804134e-06, P138=1.1140675135950518e-05, P139=1.518077690576631e-05, P140=2.2805394188954447e-05, P141=1.8505118111576587e-05, P142=8.548567339900154e-06, P143=7.197537005849411e-06, P144=5.850194539166976e-06, P145=3.556324519561382e-06, P146=2.476027588780101e-06, P147=2.5031084681432087e-06, P148=6.750939624696129e-06, P149=6.7800337918976455e-06, P150=9.950265019045342e-06, P151=1.4444825568363158e-05, P152=3.6603150124784405e-05, P153=2.169433375951953e-05, P154=2.724195398910904e-05, P155=1.6796512047074036e-05, P156=1.8328805119777513e-05, P157=7.329774254818623e-06, P158=3.0306406355948064e-06, P159=1.629814207700635e-06, P160=1.3803682252672823e-06, P161=1.457779217339001e-06, P162=2.0213330116401096e-06, P163=2.5448219241640915e-06, P164=2.6543781565250646e-06, P165=3.791396628399633e-06, P166=3.2787747553870173e-06, P167=2.997458888767311e-06, P168=1.8620527655711672e-06, P169=1.1914335083831665e-06, P170=1.5976571395054763e-06, P171=2.688529705964711e-06, P172=6.097635576947486e-06, P173=7.910049228025797e-06, P174=7.4035609533697475e-06, P175=7.219739169570783e-06, P176=7.973406973741753e-06, P177=5.090273216483867e-06, P178=1.288650976593188e-06, P179=1.2163240511612668e-07),Row( P0=-488.67982353733686, P1=32.63683120927096, P2=-21.59346035362498, P3=1.3731940808619019, P4=-12.698442457389609, P5=-19.75762185563865, P6=-4.355766847267008, P7=0.36573628837748734, P8=-3.1266500427938406, P9=-0.3510135455831599, P10=-12.783106216027095, P11=-3.0730008419453507, P12=-8.996863566056872, P13=-1.5258358918608645, P14=4.716193431116764, P15=11.383458178917255, P16=10.258709959410668, P17=17.402184983199476, P18=6.920477134255488, P19=6.574642991296477, P20=3.284875891377854, P21=0.005261798000615239, P22=4.260097041259202, P23=8.706355280483631, P24=0.9644649373721511, P25=1.3921057219308814, P26=-1.0817056454854426, P27=1.6290279464985782, P28=-0.5579577926324005, P29=4.647355231070722, P30=2.8305646432015537, P31=3.836640818509121, P32=0.8806622264099178, P33=4.586800013210305, P34=0.7043950178398277, P35=0.4659205390261467, P36=-0.9572081537404348, P37=-1.8051437542041988, P38=-0.17130390907676313, P39=2.102087267598416, P40=0.7926028125776684, P41=0.7229107677649398, P42=0.6455640207225516, P43=0.5936572216981172, P44=0.5549738765923723, P45=0.5772313272727551, P46=0.6122741714140516, P47=0.5902626488347607, P48=0.639201120905379, P49=0.6555602442558667, P50=0.691600309271573, P51=0.7483671841766766, P52=0.0037274211873449007, P53=0.0009906368779476617, P54=0.00015082845726066413, P55=0.00014432973545453016, P56=0.0006281497177890234, P57=0.0007119348481514764, P58=0.0005124016293746821, P59=0.0016160369393461084, P60=0.005925214592147162, P61=0.015587101930115626, P62=0.06301855471052895, P63=0.09641210084382074, P64=0.1136677557423999, P65=0.2644749278988862, P66=0.5098862272672001, P67=0.6929597873783565, P68=0.4322409153417803, P69=0.16863725323833537, P70=0.0867251778743995, P71=0.04523996016630938, P72=0.023360735915650473, P73=0.012031737031002007, P74=0.00783676900088611, P75=0.008491553910393301, P76=0.010787802878152343, P77=0.024700019253808624, P78=0.04121402986460716, P79=0.07715289455213753, P80=0.1707317634719472, P81=0.5169339778693339, P82=1.297675488365023, P83=1.9061231239777092, P84=4.347211139007647, P85=5.073867672012175, P86=2.2470509005166597, P87=1.4822309101577174, P88=0.8015826880349652, P89=0.12878817361402745, P90=0.12211656458801755, P91=0.11346435387314194, P92=0.09670665233490307, P93=0.3105810234034261, P94=0.6710868190407635, P95=0.41938073284028926, P96=0.3204144591804701, P97=0.28534334361967995, P98=0.16722823565330186, P99=0.11263916981330255, P100=0.07141636709992659, P101=0.019887968521084175, P102=0.02306508547622601, P103=0.038407165630731654, P104=0.06583842327370827, P105=0.1152544366652223, P106=0.1862048634394635, P107=0.07772052165919512, P108=0.08063663646449845, P109=0.1144465418696383, P110=0.07826797572084619, P111=0.05848350428477418, P112=0.03080223183266063, P113=0.017679838442368662, P114=0.01962856165543608, P115=0.015853142583951693, P116=0.04173119927271197, P117=0.05456150452740926, P118=0.04139761462872367, P119=0.05332804921780243, P120=0.027384901524297605, P121=0.030708947861584998, P122=0.026910541029856436, P123=0.03831787817911056, P124=0.0350223374215061, P125=0.007788954269092515, P126=0.013147168022042159, P127=0.0496488626311158, P128=0.045829246312775636, P129=0.03502514511322214, P130=0.03946740553915239, P131=0.07652282914100604, P132=0.024263861774023045, P133=0.014300926780764573, P134=0.010961295459533355, P135=0.006061929709805837, P136=0.009690506480103112, P137=0.019273150803945566, P138=0.009502150193332034, P139=0.024076400423198658, P140=0.024102584582490646, P141=0.013492880228887803, P142=0.01834786189380006, P143=0.010886605689578856, P144=0.0132299449391787, P145=0.00849917487190462, P146=0.003197561775302435, P147=0.001098204946371373, P148=0.0007200043355299908, P149=0.0007662347430801726, P150=0.0012201736982129982, P151=0.0019807081963625697, P152=0.0023541439362893693, P153=0.0020173129505652, P154=0.0020631692907093616, P155=0.0020059738973922567, P156=0.002150423158190194, P157=0.0014818562122107509, P158=0.0010904645826871962, P159=0.0010750816244426546, P160=0.0008919802790434269, P161=0.0006707110501104952, P162=0.001062661346409754, P163=0.0010847209954020786, P164=0.0015739129848922771, P165=0.0014355649847273293, P166=0.0013167831388820447, P167=0.0009900256922900457, P168=0.0006804813486458057, P169=0.0008175819834988784, P170=0.0007831722279602927, P171=0.0008603462199455893, P172=0.0009919212722120261, P173=0.0007768613609166025, P174=0.0006078639159886543, P175=0.0004831044842307531, P176=0.0004416890954592896, P177=0.0002790802816202738, P178=6.755261432361896e-05, P179=4.701289009730932e-06),Row( P0=-767.3827442332578, P1=82.35347603933789, P2=9.329917891359884, P3=11.137224429348043, P4=-3.770798708533598, P5=-5.293042037023497, P6=-5.0695322008621035, P7=4.758177800482778, P8=1.1506664089134984, P9=-2.274468277666821, P10=-0.8691778832132696, P11=0.7454009005762433, P12=-8.66392411731785, P13=1.7896910821042238, P14=-5.40300693537627, P15=-2.7378263871957658, P16=-0.3321733287808523, P17=-1.1954607821857186, P18=-3.9705608890674595, P19=-1.5611773973707121, P20=-3.6311901722174698, P21=-1.6694959291532294, P22=-6.403580838759207, P23=0.6528225071804432, P24=-4.894020063168731, P25=-5.248114259311279, P26=-4.989366513636588, P27=-1.793972980207857, P28=-5.37928918311503, P29=-2.925938002571376, P30=-5.299167701696521, P31=-1.843390698801488, P32=-3.297232595038573, P33=-1.5995933987591648, P34=-0.9586487536705142, P35=0.9286777408998668, P36=0.7598927668897671, P37=4.920626064071454, P38=2.7464296355538727, P39=5.530171367743121, P40=0.5341948875762899, P41=0.5783598072932402, P42=0.5695746774775338, P43=0.5319336071256264, P44=0.5349607006798349, P45=0.5560872304498788, P46=0.5779244767317204, P47=0.5919440856576579, P48=0.6219367769754346, P49=0.6086931242293144, P50=0.5831653260045809, P51=0.5252324156065088, P52=2.3850947636543726e-05, P53=1.4017142837606648e-05, P54=1.8856364270811066e-05, P55=0.0001260992337997955, P56=0.0022289270903411945, P57=0.010009383671955774, P58=0.00676524032583734, P59=0.0013645276520651126, P60=8.835493749539814e-05, P61=4.1202208780823355e-05, P62=0.00045322659855563674, P63=0.0015015908886978275, P64=0.002762130407238302, P65=0.0073066082400736415, P66=0.005524406258464263, P67=0.00063650375542335, P68=0.0002245067513835463, P69=0.00041937945071851925, P70=0.0008003157745900126, P71=0.0012293501559478136, P72=0.0021668994471415394, P73=0.0029214788601203676, P74=0.001642519199434414, P75=0.0005033853356648587, P76=0.00043475810435593214, P77=0.0017838664936136004, P78=0.0033917803818729665, P79=0.00395336742750933, P80=0.003867722493197177, P81=0.002040442464522379, P82=0.0007793070832576626, P83=0.0010647191427218207, P84=0.0016891887036924553, P85=0.002105641133457637, P86=0.0026514878540244108, P87=0.002992714781939514, P88=0.0006954492377606937, P89=0.00032512784428906404, P90=0.0002749014358217688, P91=0.0002893421264300611, P92=0.0007307677623062801, P93=0.0005056685590714916, P94=0.0001112204301166375, P95=0.00013734520225199202, P96=0.00016541652368409654, P97=0.0003398811607940879, P98=0.0001789112206705695, P99=4.3903980000001434e-05, P100=1.8933397651195256e-05, P101=7.454279559929684e-05, P102=5.853258436392894e-05, P103=4.3603352204555284e-05, P104=7.69004807651042e-05, P105=0.00015471445594277406, P106=0.00011698157566830271, P107=7.94525958451581e-05, P108=0.00016142710322404304, P109=8.964959904103167e-05, P110=2.4100748717523652e-05, P111=8.104128407489982e-05, P112=4.478408835863489e-05, P113=6.398938334440389e-06, P114=1.8838726965568392e-05, P115=1.653366215823692e-05, P116=3.8914986410239054e-05, P117=2.2937521623973616e-05, P118=1.1893816418639993e-05, P119=1.2026365771813838e-05, P120=9.958378381042231e-06, P121=9.094795147294428e-06, P122=9.44355596282337e-06, P123=8.826914951466021e-06, P124=1.1750979452582356e-05, P125=1.840119481445729e-05, P126=3.402325775427179e-05, P127=3.10507545531457e-05, P128=5.798575621567703e-05, P129=3.8728792450157346e-05, P130=1.434790052810818e-05, P131=4.954978745829856e-06, P132=4.0610597490394924e-06, P133=2.0756989498274176e-06, P134=1.7731241283836893e-06, P135=1.6861671701150188e-06, P136=1.4573671372852618e-06, P137=1.7517183451313973e-06, P138=3.159951960672069e-06, P139=4.7711114671895526e-06, P140=5.4468811989741e-06, P141=3.407527577000378e-06, P142=3.4094018976387877e-06, P143=3.398475880547235e-06, P144=3.090797939195674e-06, P145=4.385491809493801e-06, P146=1.1121029571959159e-05, P147=1.9265917480299016e-05, P148=6.135519393258674e-05, P149=0.0002016741829149586, P150=0.00043359076858510477, P151=0.0003217732543354531, P152=0.0001420015261869207, P153=4.3665172822524564e-05, P154=2.4321813321397857e-05, P155=2.1207689302141003e-05, P156=1.3695484131091853e-05, P157=1.1473214171561621e-05, P158=6.363714126072991e-06, P159=5.5055876689336785e-06, P160=7.84117205463662e-06, P161=8.223850800951315e-06, P162=8.415242080929074e-06, P163=8.897321774706743e-06, P164=5.9922329460562335e-06, P165=3.865222951902472e-06, P166=6.082848360207775e-06, P167=3.3804511630324766e-06, P168=4.3981582867570875e-06, P169=6.601029195164205e-06, P170=7.217606733187567e-06, P171=8.656622309511861e-06, P172=7.214237656996645e-06, P173=5.2148915508743395e-06, P174=5.005433134035998e-06, P175=2.705236134031043e-06, P176=2.0036078423243874e-06, P177=1.1196647542352036e-06, P178=4.877909119596291e-07, P179=3.68901684639326e-08),Row( P0=-714.2116002759158, P1=92.04624425154114, P2=10.85966686879328, P3=18.02606844918278, P4=-6.284465647716447, P5=-6.580342331727816, P6=2.427795657685254, P7=6.164820410594054, P8=0.1523449716244382, P9=-6.703579816007238, P10=-2.2112635419972535, P11=0.6768934142444962, P12=-6.10011887764957, P13=2.242335375809607, P14=-6.188423969282703, P15=-0.4742780077643711, P16=-1.1632839477326173, P17=-3.284640798647616, P18=-2.7047120482042826, P19=-4.632036902296267, P20=-4.684158769605952, P21=-4.80352945258213, P22=-7.132732258914124, P23=-0.9282193553257517, P24=-9.747884348930311, P25=-6.653536553586889, P26=-4.584219351741564, P27=-4.127538011765627, P28=-7.727357779024047, P29=-2.629105147135413, P30=-5.330870867570202, P31=-0.3780282851831589, P32=-1.8899929816640508, P33=2.0373962217761394, P34=1.098732461956533, P35=1.4768828533354896, P36=-0.7887076451418544, P37=3.085027239572069, P38=0.8250087075639851, P39=2.239834486532871, P40=0.7151821972079089, P41=0.6561264085170496, P42=0.647138607116561, P43=0.6696862886888955, P44=0.7427862409059008, P45=0.7500217212958814, P46=0.7070004529146104, P47=0.6706194799620487, P48=0.6297579348342861, P49=0.6676167480640872, P50=0.6571585649492439, P51=0.6393011175949149, P52=1.720590958895994e-05, P53=5.497216357606071e-06, P54=1.4646366660936463e-05, P55=0.00043777909854596377, P56=0.005141114349597651, P57=0.007885762268353205, P58=0.0047809670200087435, P59=0.00536358251743805, P60=0.0012778365021778781, P61=0.00018812717886425702, P62=0.0002689321912773207, P63=0.00026220911828327343, P64=0.0003777124307569314, P65=0.0015505101837088803, P66=0.005299070816879826, P67=0.011651055415048904, P68=0.004562100583236615, P69=0.00045559955737327324, P70=0.0003116529700672697, P71=0.00037281529640852254, P72=0.0007344834870004694, P73=0.0015088225873828645, P74=0.0017256817942467566, P75=0.0022913335975871993, P76=0.0015413885320806461, P77=0.0011379728863399998, P78=0.0009603561230846153, P79=0.0011443927570871147, P80=0.0015185930806135605, P81=0.0014201595758334626, P82=0.00230779317085302, P83=0.004903913093677523, P84=0.0026083815435250044, P85=0.0003433594614015543, P86=0.00010569695705315688, P87=0.00011328666374571988, P88=0.0002957271093815365, P89=0.0031305437060133506, P90=0.010229241206666477, P91=0.0024813564321623746, P92=8.375435146445805e-05, P93=5.3093202994671096e-05, P94=7.037044545091512e-05, P95=0.0011382536508003779, P96=0.0013494322088066658, P97=5.741783016309308e-05, P98=3.0239441516589148e-05, P99=8.354498247212771e-05, P100=0.0006649393633069479, P101=0.00046973595993767025, P102=1.0341003019614704e-05, P103=1.7562746165845517e-05, P104=0.00026067554010713635, P105=0.00022636342989526328, P106=2.200849993772041e-05, P107=0.0001029946787867536, P108=0.0007585480685982113, P109=0.00025271439039689327, P110=1.9135797884246414e-05, P111=0.00035252297715637645, P112=0.0005617127327039054, P113=3.0104815226205168e-05, P114=2.0700991410981578e-05, P115=6.804316325620041e-05, P116=1.5378887644328887e-05, P117=2.1402013878839358e-05, P118=1.7362997497972555e-05, P119=1.756003743964261e-05, P120=1.998130102486813e-05, P121=1.2473292506736628e-05, P122=2.753774072408799e-05, P123=9.039772002563577e-06, P124=1.2337185262421863e-05, P125=1.2182664476290249e-05, P126=4.3948353120776563e-05, P127=2.214688186024685e-05, P128=4.729497687529807e-05, P129=3.8120241458990397e-05, P130=2.2487474152317676e-05, P131=1.1697067804687261e-05, P132=7.815437952122403e-06, P133=6.741387138052659e-06, P134=3.943009445973056e-06, P135=3.879846193474693e-06, P136=5.283760715476767e-06, P137=4.908806073271622e-06, P138=1.1898690880063068e-05, P139=1.507333880867168e-05, P140=1.8629319367182342e-05, P141=2.5585749268413688e-05, P142=1.447669682754974e-05, P143=1.7508256083280865e-05, P144=1.0517081060467164e-05, P145=1.6422306791461606e-05, P146=6.0502447873489764e-05, P147=0.00013055103067457095, P148=9.980067468182054e-05, P149=7.987544462091263e-05, P150=0.0001231367657706441, P151=0.00010775309834049888, P152=0.00012560906181665517, P153=0.00010418729617148973, P154=7.42519376437239e-05, P155=4.5889217905156213e-05, P156=3.968808782310191e-05, P157=2.2709384272257497e-05, P158=1.8351951692133243e-05, P159=9.623335541587892e-06, P160=9.582419305009782e-06, P161=6.5707106743150226e-06, P162=1.205003048026166e-05, P163=1.6816261969528538e-05, P164=1.4734805925988034e-05, P165=1.640094125311281e-05, P166=1.97211889415196e-05, P167=1.8706589301441443e-05, P168=1.4118462644950916e-05, P169=1.0452677249860428e-05, P170=1.0498968834855325e-05, P171=1.0678094941332152e-05, P172=9.246644004947508e-06, P173=7.584619740572942e-06, P174=5.689414992074288e-06, P175=4.504742401857386e-06, P176=3.5265400580063996e-06, P177=2.163434525549014e-06, P178=6.324534778363586e-07, P179=4.608895115086435e-08),Row( P0=-786.7391589052852, P1=72.10555399809456, P2=16.667931402520765, P3=17.656498305005748, P4=3.338931303199799, P5=0.46566274917008554, P6=1.9680148900206806, P7=8.576410647735425, P8=7.375440075185362, P9=-0.028579638708336158, P10=3.404875758643569, P11=4.594684847501678, P12=-1.886196546616569, P13=2.314449143044059, P14=-3.321811361039023, P15=-1.3900687493587536, P16=3.1034258867885787, P17=-1.0971723915565992, P18=-1.4874970946822939, P19=-0.6402792347935135, P20=-3.0215951248402217, P21=-0.09385678702522406, P22=-4.007331919645816, P23=0.4953290184184514, P24=-2.9263325352316296, P25=-3.1714610449201035, P26=-4.864158500206989, P27=-0.7981458462619535, P28=-4.206033275005809, P29=0.37996629558036843, P30=-2.8285618905736736, P31=0.6002642624768868, P32=-3.1151080310443517, P33=0.12212216426256017, P34=-1.3155440408935142, P35=2.2254355465926885, P36=-0.900386289520857, P37=2.09598045579526, P38=0.003335040439233736, P39=4.516366354784049, P40=0.7034399624928906, P41=0.7240441882413393, P42=0.7566423472206594, P43=0.6990246202140515, P44=0.7055570877649245, P45=0.7452531103174188, P46=0.8095014456585135, P47=0.8620096780390631, P48=0.8249293355688662, P49=0.7711810071766819, P50=0.7341340489224363, P51=0.7234257948319067, P52=0.00015071366119631438, P53=4.354858994240423e-06, P54=3.5430934359454557e-06, P55=1.9985707865004835e-05, P56=0.0017265338165911065, P57=0.008880546696767029, P58=0.003748383705601328, P59=0.0006031951990420137, P60=9.689687184856963e-05, P61=5.2982677230501064e-05, P62=0.0005013551030528617, P63=0.001435949976507797, P64=0.0008734405755388413, P65=0.0006460256760789218, P66=0.0003304231781871881, P67=0.0001399071773795508, P68=0.0002375494948320887, P69=0.0004956397269351387, P70=0.0004529286718039861, P71=0.00027668522209831276, P72=0.00021979329970671705, P73=0.0001407017727573013, P74=0.00021205041671042043, P75=0.00039508767277656137, P76=0.0005584930967363615, P77=0.0009986038707194196, P78=0.0016025078518381358, P79=0.001511501747361787, P80=0.000581214693082928, P81=0.0004340979582449911, P82=0.00039875251717692964, P83=0.00013029166968464077, P84=8.053020420991999e-05, P85=0.0001474786870178349, P86=0.0002510877710065702, P87=0.00040301624122318885, P88=0.00019350375855010644, P89=2.4089523017312527e-05, P90=1.980771234121614e-05, P91=6.811249744357457e-05, P92=9.973892754361293e-05, P93=2.0152168476846328e-05, P94=1.2818024639755266e-05, P95=2.2382967119577486e-05, P96=4.921757421640337e-05, P97=1.9705915680914548e-05, P98=1.1179464520860571e-05, P99=9.419851480006246e-06, P100=8.315132944994763e-06, P101=4.1847313849551815e-06, P102=3.055991037697588e-06, P103=4.055555156661597e-06, P104=5.510015961437991e-06, P105=5.0295987315378395e-06, P106=4.418801147345245e-06, P107=1.9192098364998383e-06, P108=2.8013330288248743e-06, P109=3.692249920682641e-06, P110=2.003359198441356e-05, P111=4.268851652642948e-05, P112=8.892770599732278e-06, P113=3.7409733049020016e-06, P114=5.00974954613628e-06, P115=1.4934099909174414e-05, P116=1.0926176271005256e-05, P117=5.269240469115457e-06, P118=6.222867742290055e-06, P119=2.8179841752176885e-05, P120=2.2350931571709842e-05, P121=4.190340049997006e-06, P122=6.305901861160087e-06, P123=5.066964097769788e-06, P124=4.709324926592704e-06, P125=5.024447545355566e-06, P126=6.5444336199990225e-06, P127=8.652240215421215e-06, P128=1.1557067191982372e-05, P129=1.4484187528745403e-05, P130=6.0182687561974495e-06, P131=3.7522960382018614e-06, P132=2.5432153924115424e-06, P133=1.5570950838941438e-06, P134=1.7137006054257712e-06, P135=1.2675741408611163e-06, P136=1.1372452116296602e-06, P137=9.76411373975328e-07, P138=1.1919500222228041e-06, P139=2.083173892455612e-06, P140=4.037838328809801e-06, P141=2.7955088859433716e-06, P142=1.1804789914869085e-06, P143=2.21833555978649e-06, P144=4.7579186729595644e-06, P145=7.16394338627738e-06, P146=1.984566274100121e-05, P147=4.981379706829099e-05, P148=7.920334147014644e-05, P149=8.54041200675714e-05, P150=0.00010106227385029052, P151=4.751911795146551e-05, P152=2.0286683230690257e-05, P153=1.1808487712838271e-05, P154=1.0846324032773975e-05, P155=6.889873607864664e-06, P156=6.989220024750359e-06, P157=7.944363642478926e-06, P158=6.330690126206579e-06, P159=5.866731368749166e-06, P160=7.822797927421679e-06, P161=9.559293072179571e-06, P162=4.934452122815815e-06, P163=3.5521977188785943e-06, P164=3.190892641739979e-06, P165=3.2573836444348124e-06, P166=2.8155662083774226e-06, P167=2.6151792153488586e-06, P168=2.2799932119976314e-06, P169=2.90687915621824e-06, P170=3.1477560371885324e-06, P171=2.132339787428067e-06, P172=9.04378356715129e-07, P173=1.2344927680018567e-06, P174=2.2108212368113907e-06, P175=4.196861164638104e-06, P176=4.37449698548527e-06, P177=3.0264451313188874e-06, P178=1.5441179779194304e-06, P179=1.304821857967004e-07),Row( P0=-816.9443200654546, P1=76.92855696152313, P2=19.070137786528928, P3=15.12757653726522, P4=1.4550526316207406, P5=-0.1713258810181111, P6=3.0062222667996985, P7=11.186598682702797, P8=3.269519141082597, P9=-5.205710115951302, P10=1.8821605709165228, P11=1.1353504095879092, P12=-3.9477671442238633, P13=4.113093840980437, P14=-3.9169149076343928, P15=0.41860091733481425, P16=0.8324043208791956, P17=-1.16675263824163, P18=-2.5913979434705094, P19=-2.220891848100299, P20=-4.677890145637639, P21=-1.204813806419169, P22=-5.965583985177583, P23=1.0261772400521436, P24=-7.399667820570247, P25=-4.519225515467907, P26=-4.873429825053841, P27=-2.3625025227518552, P28=-4.572940008104331, P29=-0.9392322293755354, P30=-3.030399139071687, P31=-0.07226025102909381, P32=-3.288708265198748, P33=-1.9908183993173094, P34=-0.8542005819526851, P35=-1.2257749839967478, P36=-2.26127063995116, P37=-0.061089782279200776, P38=-0.8209220569429273, P39=2.3953249232217155, P40=0.7276396786020666, P41=0.7379431949871452, P42=0.7593424069780258, P43=0.7792371654475421, P44=0.8068665163428926, P45=0.8025202408954721, P46=0.8010092525521552, P47=0.8029640493999468, P48=0.7456719267196031, P49=0.7153297660191673, P50=0.6644569542733431, P51=0.6594034427813281, P52=1.1137198140807816e-05, P53=1.1219924516192725e-05, P54=1.0069642955233103e-05, P55=0.00018175066287558428, P56=0.0025114112266987334, P57=0.00496865269388833, P58=0.0014024327683083227, P59=0.00015503346824595376, P60=5.2467925603942645e-05, P61=5.508049244416535e-05, P62=0.0001312549902702128, P63=0.00022980264986194958, P64=0.00024708258344047977, P65=0.0001833597648591752, P66=9.970732340956318e-05, P67=0.00015073657523907163, P68=0.00023333053884611377, P69=0.0004150892643230077, P70=0.00032008769470086686, P71=0.0002677313191585482, P72=0.0001409869394121821, P73=9.067114659382914e-05, P74=0.0001222841653730653, P75=0.00020770454558496698, P76=0.00024813740660819807, P77=0.0007324816589770722, P78=0.0009153886138272072, P79=0.0004531965738214999, P80=7.572699049548313e-05, P81=3.5019542560356856e-05, P82=4.930629212537339e-05, P83=0.0001109662841656167, P84=0.00020629951954716962, P85=0.00014622072997300017, P86=6.234878789008394e-05, P87=4.120393917657143e-05, P88=3.6692013732864164e-05, P89=3.742761225803532e-05, P90=4.1525747027331026e-05, P91=2.3002038633725424e-05, P92=1.5296658833224715e-05, P93=6.699459234732482e-06, P94=1.4029724089988256e-05, P95=2.2710369139634774e-05, P96=8.654416993496096e-06, P97=7.001300112605954e-06, P98=6.546913161046562e-06, P99=4.922209108597345e-06, P100=2.754820796656833e-06, P101=1.4567271871452587e-06, P102=1.2092473161367377e-06, P103=1.1257614347352023e-06, P104=9.977929044656344e-07, P105=7.619578258064508e-07, P106=7.625085308550752e-07, P107=1.1979667769631779e-06, P108=4.4229011669141676e-07, P109=3.417656504149431e-07, P110=5.993726240000947e-07, P111=3.620871465658826e-06, P112=1.9263551418392607e-06, P113=5.147871921169137e-07, P114=2.5758390762467827e-07, P115=4.929064042555823e-07, P116=1.5679281465188368e-06, P117=1.1420475717956135e-06, P118=1.411007082417305e-06, P119=2.191642004663407e-06, P120=4.001223029961818e-06, P121=1.6978070384789844e-06, P122=6.4905664806929624e-06, P123=1.5566748469347234e-06, P124=1.7169566790425155e-06, P125=2.0020021154233176e-06, P126=1.951184740974678e-06, P127=2.648772549267584e-06, P128=3.913928167692682e-06, P129=4.391072013686838e-06, P130=3.4024814927929463e-06, P131=2.4771986340053097e-06, P132=1.2622898068026879e-06, P133=7.740429469385707e-07, P134=6.778088373146575e-07, P135=5.932332603416692e-07, P136=2.337725058650639e-07, P137=1.6195498189135533e-07, P138=1.7879989613306843e-07, P139=1.4391734652642644e-07, P140=1.778610350058085e-07, P141=2.1262069946112683e-07, P142=3.2695677626171234e-07, P143=2.874884271166157e-07, P144=1.9520846662675224e-07, P145=2.6176003606470205e-07, P146=6.049468263184101e-07, P147=6.043310590963179e-07, P148=2.5393425715043386e-06, P149=6.517555027158382e-06, P150=5.131508178350196e-06, P151=7.57990314398382e-06, P152=7.959808421019912e-06, P153=5.938878621972619e-06, P154=5.691870595708189e-06, P155=7.678255650252505e-06, P156=6.979858677668497e-06, P157=2.1690906518016152e-06, P158=1.2921529418554041e-06, P159=1.3146187313080934e-06, P160=1.217493053781652e-06, P161=1.5338024586766085e-06, P162=1.2549460012479898e-06, P163=1.2511838708974366e-06, P164=1.136754390609327e-06, P165=8.450788361808967e-07, P166=3.9991341867091293e-07, P167=3.4246009544780195e-07, P168=3.1210071673039905e-07, P169=5.729800109706876e-07, P170=9.329822311793528e-07, P171=1.6912733845064745e-06, P172=1.9771755030295586e-06, P173=1.8314066387041365e-06, P174=2.6647727396919728e-06, P175=1.8179090124297312e-06, P176=1.1127582236108841e-06, P177=7.277260614880656e-07, P178=6.42493089034481e-07, P179=7.051155454524065e-08),Row( P0=-787.4721943299402, P1=95.65707573085622, P2=24.00034986691855, P3=22.362216620147493, P4=-1.2124677645188733, P5=-3.4284854678319436, P6=-0.3144636986380837, P7=9.69578961826287, P8=1.5257383243368148, P9=-3.8656095816451628, P10=2.1697907858706786, P11=0.22940398384599764, P12=-3.4102624655012606, P13=4.132454662900964, P14=-7.335240915283002, P15=-2.2326180556780097, P16=-4.064889029519424, P17=-6.610932208035754, P18=-3.928445529632955, P19=-4.336663050153249, P20=-5.962831221302418, P21=-3.6838611406788493, P22=-8.383835521238789, P23=-1.7450387815204789, P24=-9.360819909042592, P25=-5.729725843167387, P26=-3.736391681503545, P27=-3.4744020674114493, P28=-7.264006389892163, P29=-1.7148845573085236, P30=-4.481819992820663, P31=-0.5909698465920604, P32=-3.6187888318354644, P33=-3.213017417817008, P34=-2.8107681944390235, P35=-2.6447846993095525, P36=-3.1407506102628315, P37=0.4793032613382686, P38=-2.5539521713890596, P39=0.14300412246466987, P40=0.6682783775602088, P41=0.676172161990443, P42=0.7281175208107451, P43=0.7701264103197986, P44=0.8390789711073716, P45=0.8348959659171645, P46=0.8124248814689589, P47=0.7289828146724293, P48=0.6902207904112084, P49=0.638325693818504, P50=0.567075778831852, P51=0.5996994983021271, P52=5.944346602727714e-06, P53=2.4156113598346803e-06, P54=5.68261065833149e-06, P55=0.00022148749317636252, P56=0.004401029604437569, P57=0.009613938126880689, P58=0.00289644460301846, P59=0.00019418167723393747, P60=4.592659691530088e-05, P61=0.00010132663516868762, P62=0.00017137967393685536, P63=0.00016034334419160164, P64=0.0001025288489638732, P65=5.4972290378936094e-05, P66=5.236249956395283e-05, P67=0.00010626253280426091, P68=0.0002028474051213269, P69=0.0006001892139635124, P70=0.0008540662261423447, P71=0.0003237600468974553, P72=5.808547844937668e-05, P73=8.315845951691469e-05, P74=0.00016220516630957016, P75=0.0003559819519859243, P76=0.00030751705754933925, P77=0.00027979729612210186, P78=0.00016009316619935963, P79=8.480115429437333e-05, P80=5.124374863872212e-05, P81=4.703001348082836e-05, P82=6.359207091529154e-05, P83=7.547505119546394e-05, P84=9.097641808353515e-05, P85=8.157225728866224e-05, P86=9.77122955614314e-05, P87=0.00023826965372898391, P88=0.00024682583410179837, P89=8.345432902723087e-05, P90=0.0001064511621131598, P91=0.00016394239889051, P92=0.00032969879471846444, P93=9.84569841855023e-05, P94=2.629685364776695e-05, P95=3.3389076488889924e-05, P96=8.943586649070651e-06, P97=7.332231121038167e-06, P98=5.977058245826208e-06, P99=5.250337649728815e-06, P100=8.229738846820077e-06, P101=2.9139491130760546e-06, P102=3.0076911143222547e-06, P103=2.205475605566132e-06, P104=7.650456951024261e-07, P105=9.541988864434693e-07, P106=9.181234967949673e-07, P107=1.5397288553685123e-06, P108=1.511254774403274e-06, P109=1.0817965274168372e-06, P110=1.9988019346031805e-06, P111=5.356516009219589e-06, P112=3.0886615164503195e-06, P113=1.8990550857167474e-06, P114=6.026673596513602e-07, P115=1.416228854116326e-06, P116=1.6701607911624167e-06, P117=2.138729043240266e-06, P118=1.4893999693371476e-06, P119=1.2591946188263513e-06, P120=9.70885806770421e-07, P121=1.1267566556345864e-06, P122=1.665062331749737e-06, P123=9.085025322105918e-07, P124=5.61890878443516e-07, P125=1.451092374306664e-06, P126=1.9901879615509486e-06, P127=2.368022496698673e-06, P128=2.3142231866358226e-06, P129=3.0237639032781273e-06, P130=1.927105643305296e-06, P131=1.5533098189659408e-06, P132=1.0194761420989536e-06, P133=7.541413778221839e-07, P134=5.541988123169337e-07, P135=4.2085396715717814e-07, P136=5.294921339209031e-07, P137=3.70664519354051e-07, P138=6.715497920320526e-07, P139=5.139499379512626e-07, P140=4.365650925331653e-07, P141=7.537179610468604e-07, P142=4.0269403152176946e-07, P143=3.62642186836679e-07, P144=1.3926362743060325e-07, P145=1.613928936447591e-07, P146=2.927789115023319e-07, P147=6.554078832602489e-07, P148=1.112177178828415e-06, P149=1.4548192632040351e-06, P150=2.924900618838563e-06, P151=3.5172643457460437e-06, P152=5.580348489919146e-06, P153=2.3645174854510715e-06, P154=2.572380154720715e-06, P155=6.600920017947864e-06, P156=1.8521475829668613e-05, P157=6.083661118203901e-06, P158=1.9865165267131827e-06, P159=1.2214963410889351e-06, P160=1.1621419726931714e-06, P161=1.2222838659018787e-06, P162=8.228370048266896e-07, P163=1.5752328254771092e-06, P164=9.907582942675604e-07, P165=6.139014445776923e-07, P166=4.1498010455217467e-07, P167=2.686150053562573e-07, P168=2.1675902539132088e-07, P169=3.3869790311796935e-07, P170=3.9057481124914184e-07, P171=4.4172232619402394e-07, P172=3.872243122925607e-07, P173=3.1440735671862853e-07, P174=2.252339853831257e-07, P175=2.46469361296646e-07, P176=2.9212693492497317e-07, P177=3.1733154271438847e-07, P178=2.2347951929085043e-07, P179=3.077004985521284e-08),Row( P0=-655.4439854306594, P1=82.88236788841904, P2=-18.10949642537295, P3=10.68145524442761, P4=-8.39394528572807, P5=-14.648943398633511, P6=0.15072484106031142, P7=-4.6745377903226455, P8=-2.4253986738840445, P9=-1.833934905460597, P10=-6.521627292848515, P11=-3.409731197191668, P12=-5.720471162098081, P13=-9.064167967577811, P14=-1.8170710206189273, P15=-2.1877119310158144, P16=-4.428860953543679, P17=-1.877893010790258, P18=-3.8231426594126683, P19=-5.112241974366994, P20=-0.522372766536754, P21=-4.25129555639098, P22=-5.454317920241926, P23=-4.555312472108556, P24=-4.644562935049288, P25=-5.360858064251166, P26=-6.31381643171113, P27=-2.239917108158376, P28=-6.345338537050055, P29=-3.6051381010845, P30=-2.0064638618426422, P31=0.9059688731366355, P32=-0.5473001108590824, P33=2.79927890206946, P34=-0.9680344136506215, P35=0.0816679734791516, P36=-1.0598326869291286, P37=1.7644184735455246, P38=1.8705299654928897, P39=4.830747478198571, P40=0.7339414660726212, P41=0.6383535482035404, P42=0.6405026360617103, P43=0.6428695672969161, P44=0.6616018583500283, P45=0.6647376389802822, P46=0.6753758303542029, P47=0.6961326231865803, P48=0.7137692410936107, P49=0.6982681836513163, P50=0.7181812364253461, P51=0.7409270097529618, P52=8.166356531594501e-06, P53=4.200284466485153e-06, P54=3.8377128906405505e-06, P55=7.471885676216088e-05, P56=0.0007739805774409266, P57=0.004316137921673106, P58=0.004720048659879623, P59=0.0036856777640542704, P60=0.0012755013604480126, P61=0.0001441737196200272, P62=0.0007544033213435282, P63=0.00242616299748815, P64=0.003738224556598898, P65=0.006930591199329754, P66=0.012325792054048027, P67=0.011623852407515775, P68=0.0072099537654839166, P69=0.0024124072961344044, P70=0.003700858853075208, P71=0.002490363031400559, P72=0.001976877333728343, P73=0.006080064202383311, P74=0.00723788243581409, P75=0.006873252108612998, P76=0.009689665717848442, P77=0.014980486209878182, P78=0.008147620384294436, P79=0.00306260595282382, P80=0.0032421690374401266, P81=0.0055429154275192915, P82=0.009483062583589441, P83=0.018707420657248435, P84=0.04372271982027549, P85=0.039282739554399385, P86=0.006145547294663494, P87=0.003145396560475632, P88=0.0024021071575974187, P89=0.010484043855158085, P90=0.04012675404520896, P91=0.07559768889638142, P92=0.030726941855041955, P93=0.0011408299878219914, P94=0.0005304414338991885, P95=0.0015249086216960576, P96=0.0014290232420376798, P97=0.0038243955677906075, P98=0.001757335987396792, P99=0.0005229032205574289, P100=0.0009968282218634765, P101=0.0021471852571950042, P102=0.0022694258831214486, P103=0.0005384925800142706, P104=0.0014855155840161017, P105=0.004573830283398686, P106=0.0068195343942034905, P107=0.001762589965020785, P108=0.003493444180373353, P109=0.005262359867896041, P110=0.0029104973849017357, P111=0.0011706193232842736, P112=0.0009091636884806878, P113=0.00055270262497395, P114=0.00020519828373305015, P115=0.0002810395726308525, P116=0.0004939101660500746, P117=0.00025139048274627014, P118=0.00024794030799002567, P119=0.00017868879206214128, P120=0.00029594806914350123, P121=0.0003277233341104942, P122=0.0002042757016297883, P123=0.00040264231745581455, P124=0.00039662002975944993, P125=0.00041635238562751726, P126=0.0003122671774884309, P127=0.0003992268642639997, P128=0.00035854360622350755, P129=0.00014031277114239662, P130=7.948014352177817e-05, P131=4.373776158678975e-05, P132=3.367543870568482e-05, P133=3.671127298333693e-05, P134=5.774432743923167e-05, P135=0.00010912028934137237, P136=0.0001981973582932626, P137=0.0006405117518398085, P138=0.0011178452302974702, P139=0.0006941729100753321, P140=0.0004927617458250342, P141=0.00020375852955761088, P142=0.00013492123973800614, P143=0.00010391151842981842, P144=9.12182127973705e-05, P145=5.051396829773621e-05, P146=4.606826189877147e-05, P147=4.961127195467409e-05, P148=5.663053339747282e-05, P149=4.488118646443798e-05, P150=5.8237641099698934e-05, P151=5.108152942241598e-05, P152=8.778480008594948e-05, P153=0.00012676057209108735, P154=0.00010857642664208945, P155=0.0001226009731932208, P156=9.172134111636357e-05, P157=8.731921186902055e-05, P158=6.595458849261908e-05, P159=4.417214139700597e-05, P160=2.9008443549600237e-05, P161=3.314319446054681e-05, P162=2.1680688442757584e-05, P163=9.506117294615135e-06, P164=5.604950285808775e-06, P165=5.116314423467223e-06, P166=3.279147530699904e-06, P167=5.0516865101293045e-06, P168=5.6610200494507205e-06, P169=4.720869501936666e-06, P170=4.727765536408554e-06, P171=2.385173292258708e-06, P172=6.201275592860981e-07, P173=7.053083762812978e-07, P174=2.1201097647419336e-06, P175=4.567524948268373e-06, P176=6.3159564833054045e-06, P177=5.735895702595397e-06, P178=3.0671675228118313e-06, P179=3.2148313369406645e-07),Row( P0=-810.4754696574565, P1=63.6549273065988, P2=13.528320428492645, P3=12.584766146442952, P4=-6.314608214748455, P5=-4.644867510883667, P6=-2.1969893193356786, P7=2.473204236411875, P8=3.637803392262604, P9=-4.0903725564419915, P10=-2.2973741028746866, P11=-0.011845553497205352, P12=-3.449217502501548, P13=-2.019988988399138, P14=-5.574746239364075, P15=-2.4383198211060493, P16=-0.7859472855362588, P17=-3.150653637322486, P18=-4.46535420630306, P19=-4.540415472075051, P20=-4.096801484438292, P21=-3.076549711310688, P22=-6.883716510048437, P23=-1.4194745985598038, P24=-7.745054433996509, P25=-4.549160822059091, P26=-5.255662569490195, P27=-2.4974034148467914, P28=-5.89519329953755, P29=-1.3297338816684938, P30=-3.778818226465042, P31=-1.22752879484003, P32=-3.6337986412013277, P33=-0.9068572060914165, P34=-2.3787776347617857, P35=-0.6196343656134156, P36=-2.0005472191661102, P37=1.413303501364433, P38=1.0309469289008386, P39=4.872536859636657, P40=0.7207670545548737, P41=0.7349716857942529, P42=0.7605065701483056, P43=0.7611487231209023, P44=0.7362837523971851, P45=0.721631676661831, P46=0.7645507714468267, P47=0.8100355657241084, P48=0.8473220457417514, P49=0.8355032957672119, P50=0.7649793907862847, P51=0.7073746257177775, P52=1.3329501438735452e-06, P53=5.843558973683616e-07, P54=2.4929649285699383e-06, P55=3.5879947388881144e-05, P56=0.0015332779779801634, P57=0.009834870414079544, P58=0.004965414008526531, P59=0.0006022914409845065, P60=9.388908682174843e-05, P61=5.595964531982353e-05, P62=0.00011474052865081497, P63=0.0003487729705218072, P64=0.0006014444606327374, P65=0.0006783862604606153, P66=0.00030101514611306193, P67=0.00011139166098697908, P68=0.00016786190945789685, P69=0.0005564924106923609, P70=0.0008139971866204169, P71=0.00042582191208919985, P72=0.0001804713135552055, P73=0.00011954425186306048, P74=0.00012108988490134989, P75=0.00021822413638245713, P76=0.00022657448548283788, P77=0.0004127115349459425, P78=0.0005484787201639062, P79=0.0003836429575737473, P80=0.00020928174336506904, P81=0.00017113223808121392, P82=0.0001778036689139607, P83=0.00029312963440275737, P84=0.0005698880041155283, P85=0.0003944577563149595, P86=0.00015709570344123845, P87=0.00015137222630292807, P88=0.0001609690299481152, P89=0.00015097090265633303, P90=0.0002119430570080875, P91=0.00011010903213449545, P92=5.8955213441092146e-05, P93=2.7530813240142006e-05, P94=5.6149574034065846e-05, P95=5.341731335343783e-05, P96=1.901061545666417e-05, P97=1.678007974348948e-05, P98=3.1990054394168684e-05, P99=2.204626246296682e-05, P100=9.398189229368032e-06, P101=3.9643896971825075e-06, P102=4.197907028917018e-06, P103=3.691403397293776e-06, P104=3.1097777972918046e-06, P105=3.327593210924261e-06, P106=2.7928267781990283e-06, P107=1.71307397754639e-06, P108=1.1775669403562046e-06, P109=1.4526945844683939e-06, P110=6.301792082980274e-06, P111=1.4662239831237716e-05, P112=6.052147657881133e-06, P113=3.3619956450828097e-06, P114=7.525663265943179e-06, P115=7.727819133277071e-06, P116=6.251337802631057e-06, P117=3.1892182587253806e-06, P118=3.5745337634333696e-06, P119=7.85015637064393e-06, P120=1.1411657912433349e-05, P121=3.595267588041787e-06, P122=3.904048087778274e-06, P123=2.285724826106139e-06, P124=1.7329488098904045e-06, P125=1.3418939416553536e-06, P126=1.6730584697427337e-06, P127=3.3197982510167654e-06, P128=2.940369299757893e-06, P129=3.4592906419767623e-06, P130=3.2183068376783195e-06, P131=3.490059612966306e-06, P132=2.4382023400098944e-06, P133=1.8870289224849357e-06, P134=2.8533126033438843e-06, P135=1.6690177807879161e-06, P136=1.377737257219008e-06, P137=1.4247255733933837e-06, P138=1.9482460926751583e-06, P139=2.937670028852268e-06, P140=2.7811943926758143e-06, P141=2.579116846681684e-06, P142=2.694456927807079e-06, P143=2.6709982868135534e-06, P144=4.369731149613998e-06, P145=4.937747070986102e-06, P146=1.0777183256394294e-05, P147=6.599001801082121e-05, P148=0.0001052196091191484, P149=6.617088441659882e-05, P150=4.999653407243162e-05, P151=3.856074636403449e-05, P152=1.906365538477258e-05, P153=8.72810781711958e-06, P154=6.845647381647529e-06, P155=4.263512290550887e-06, P156=4.7454225367878846e-06, P157=4.156332059264263e-06, P158=3.647229394597879e-06, P159=2.841402882065035e-06, P160=2.3121663723872343e-06, P161=2.2424911813397163e-06, P162=2.648789468508032e-06, P163=1.6215865105885545e-06, P164=1.80229311385298e-06, P165=1.1637598693773635e-06, P166=6.748839675013113e-07, P167=9.841379874541442e-07, P168=1.3477554838202657e-06, P169=1.2157796737982192e-06, P170=7.559141448946284e-07, P171=7.50699594018952e-07, P172=7.94873082086128e-07, P173=8.669241926693108e-07, P174=1.3614349792363363e-06, P175=2.508197177744904e-06, P176=3.4291758116728433e-06, P177=2.476075065014911e-06, P178=8.429116642403305e-07, P179=8.920311836768911e-08),Row( P0=-669.3705367024405, P1=72.7992321523115, P2=-10.89404489179157, P3=14.393582768925647, P4=-11.086585526473506, P5=-12.83549137309659, P6=-3.6598947115523424, P7=-6.574186009755348, P8=-2.607748431948074, P9=-4.332465528300087, P10=-4.9706186328962785, P11=-5.71763645274361, P12=-7.062656138978529, P13=-8.421045423408925, P14=-5.108760325073038, P15=-5.634678672752196, P16=-3.2716876604410383, P17=-1.1560143153561622, P18=-5.4335232393949005, P19=-4.615982444033446, P20=-0.4705034544599399, P21=-4.497489357964862, P22=-6.604381137769472, P23=-1.9334478296429116, P24=-6.29248292475487, P25=-2.9455898247847747, P26=-6.1923952563959315, P27=-2.687158386728484, P28=-4.731287066666227, P29=-1.2934539085328036, P30=-0.4628294205854424, P31=1.3222496059679714, P32=-2.45352304197619, P33=0.4269911602987993, P34=-3.9923343902094874, P35=-3.239571946408887, P36=-1.760952086448675, P37=1.0081276438951836, P38=0.5880614879172283, P39=3.6578722147478837, P40=0.7464746860184799, P41=0.7174243285144318, P42=0.6995536897088619, P43=0.658054798459705, P44=0.6784292648661968, P45=0.7037086270554144, P46=0.7364433736278981, P47=0.7261057642289946, P48=0.7674452060085293, P49=0.7660195458388493, P50=0.7361189657867621, P51=0.736271634325042, P52=1.2494738282598786e-06, P53=7.044867430499302e-07, P54=4.953009409445495e-06, P55=3.373653224578977e-05, P56=0.0005682721571817124, P57=0.003773245717872065, P58=0.003371777428002792, P59=0.001994692642560821, P60=0.0024763868186148206, P61=0.0008214378213384445, P62=0.001272308965910773, P63=0.0028651622523435346, P64=0.003761876354810082, P65=0.007644892643610111, P66=0.005408095858375327, P67=0.002047974273816698, P68=0.005426339732855575, P69=0.009519607315682055, P70=0.005307955889878628, P71=0.002789403223435438, P72=0.0017615422170847276, P73=0.0019555168217133723, P74=0.001502827434226102, P75=0.0028942232382535176, P76=0.005109583735185443, P77=0.01946594927473738, P78=0.021330283652071215, P79=0.008465421400490161, P80=0.004055464339855714, P81=0.004132018960424534, P82=0.00577215720879148, P83=0.00930071684520478, P84=0.017268541498427322, P85=0.027730619697593942, P86=0.030354935252627446, P87=0.026677926917274955, P88=0.0036181824794182583, P89=0.0012393270900965163, P90=0.0013674909382234957, P91=0.004314788574645983, P92=0.02835511038608462, P93=0.03789758615580318, P94=0.03445774757303217, P95=0.0014330390527399867, P96=0.0001142098813680127, P97=0.001178024997309072, P98=0.002743198201580876, P99=0.005568898199704296, P100=0.0014811031471394025, P101=0.0004245896187351835, P102=0.0007030740681754171, P103=0.000722109789681243, P104=0.0017184702304052, P105=0.00034372804499454315, P106=0.0006216965843178557, P107=0.0013454002802716807, P108=0.0020467210226916967, P109=0.00044565514541593785, P110=0.0007347299722010666, P111=0.0021097666347502796, P112=0.0035408928190463255, P113=0.00042935478394560527, P114=0.0004144310352156098, P115=0.0007976327048406781, P116=0.0003323546323855663, P117=0.00033914403737523083, P118=0.0002170118668588485, P119=0.00014057645548667117, P120=0.00042364664443058915, P121=0.00020459385165511957, P122=0.0002748423187545423, P123=0.00030040196207323314, P124=0.0006456713467071216, P125=0.0012685567763874275, P126=0.00027291836434266474, P127=0.00289991365546963, P128=0.0014711174294284543, P129=0.0012163946075580802, P130=0.0011497443021094293, P131=0.0010601631002181264, P132=0.00038355631781296427, P133=0.000506028231621069, P134=0.0006323129332761327, P135=0.0008861410631704179, P136=0.0013115121843013965, P137=0.002532214856623343, P138=0.0024130487412595512, P139=0.0036947813232908735, P140=0.0016680552707417935, P141=0.0007184098467671812, P142=0.0003211976658479875, P143=0.00019117678352932068, P144=0.0001308151301720806, P145=6.63211933791438e-05, P146=5.107593215095091e-05, P147=3.9399647788883624e-05, P148=3.2338837681064916e-05, P149=5.738508429351716e-05, P150=0.00016313122679747927, P151=0.0002448366349210554, P152=0.0001437857955729168, P153=5.4686697133422635e-05, P154=4.9399431957012774e-05, P155=4.3188246972116186e-05, P156=4.592793737996775e-05, P157=3.5709749101846397e-05, P158=2.770490796633247e-05, P159=2.4410867686699826e-05, P160=2.733491256585806e-05, P161=2.2573643976148268e-05, P162=1.4204669737908364e-05, P163=1.3839389404860129e-05, P164=1.0014015030634817e-05, P165=1.0760901324855284e-05, P166=8.330912574475536e-06, P167=6.3006840698405584e-06, P168=5.4925872405572274e-06, P169=3.1617735545636783e-06, P170=2.0627705609994454e-06, P171=1.8499531158564383e-06, P172=4.5952206021767746e-06, P173=5.197087123260092e-06, P174=3.778023727281063e-06, P175=3.957197734385081e-06, P176=5.654709676909928e-06, P177=4.491785551171182e-06, P178=2.0910422942147833e-06, P179=1.8580268973353376e-07),Row( P0=-706.575848683725, P1=104.57278694948246, P2=0.7861521016154733, P3=9.333441085839322, P4=-4.6831856850479845, P5=-17.90410465331034, P6=-6.052370500286884, P7=1.5252454386416163, P8=-0.24093554731413944, P9=-2.4729959412473894, P10=-11.409909750909826, P11=-5.51626791040017, P12=-7.126509563125181, P13=-5.509529240012989, P14=-4.771678959364343, P15=-5.211154810350686, P16=-2.328753358545738, P17=2.0715210600694274, P18=-4.2004627260815885, P19=-2.1224114867769783, P20=-0.41713970146824164, P21=-2.3984797191126366, P22=-1.0236888529004053, P23=1.2986231401681607, P24=-3.837728033295982, P25=-1.5505789891374102, P26=-5.476173746610412, P27=-1.8035416081140545, P28=-5.707626204210113, P29=-1.1875388317478435, P30=-2.8481016550402676, P31=1.0226050600472993, P32=-0.5974441257876445, P33=0.581820456883014, P34=-1.3930831850783987, P35=0.43116221708971103, P36=0.02942016124390692, P37=3.4314769693858382, P38=3.259391092249789, P39=6.37204923013237, P40=0.7227031440435224, P41=0.6762513536599183, P42=0.6396380116920488, P43=0.6503845236097944, P44=0.6173314213559367, P45=0.6432719987592174, P46=0.6718194446053481, P47=0.6896054738074309, P48=0.7418595502419942, P49=0.7313316197380693, P50=0.6814749476568324, P51=0.6812598961518949, P52=1.1002210778008876e-05, P53=3.5808238623737398e-06, P54=2.671166678878165e-06, P55=1.9776044658901504e-05, P56=0.0006010101493967293, P57=0.005559151724038974, P58=0.00575202643030693, P59=0.003993046953369954, P60=0.0017811535373142293, P61=0.0005089048570934029, P62=0.0009682475663132074, P63=0.003020153396129704, P64=0.004678191498574572, P65=0.005392480116643373, P66=0.0054273316919419435, P67=0.003974301087003712, P68=0.0023717705291160292, P69=0.0012386664972532152, P70=0.0016323980733073265, P71=0.0015353778737833786, P72=0.0016470063124400056, P73=0.0016239309729562622, P74=0.0017690427401252716, P75=0.004095718486565191, P76=0.006071974430293423, P77=0.007584887799062588, P78=0.005420018598472503, P79=0.004137063195218494, P80=0.0023814570933094036, P81=0.0021119576622278943, P82=0.003929615881958223, P83=0.012053347646189224, P84=0.020186035445089883, P85=0.009665227241184082, P86=0.003078089313229609, P87=0.00510402635847628, P88=0.005198223223045408, P89=0.004264628361971725, P90=0.0019933225752550884, P91=0.0008354085544056114, P92=0.0024595800798203112, P93=0.0025925769748211818, P94=0.0021481525668755478, P95=0.0006218033063734956, P96=0.00011791859440083073, P97=0.0003213767660483171, P98=0.0005108111265080639, P99=0.00022335209549281283, P100=6.671206059319239e-05, P101=6.72787067699132e-05, P102=0.0001126246588543313, P103=6.262568557353917e-05, P104=4.55838915097988e-05, P105=9.047864924767875e-05, P106=0.00013274446631146727, P107=8.785436096143571e-05, P108=8.972261910855963e-05, P109=0.0002038814490719856, P110=0.0002582021493056553, P111=0.00019773198070783597, P112=9.133877849387749e-05, P113=0.00010326902838955178, P114=0.0001590389415889916, P115=0.0001671331329182015, P116=0.00012620780204117973, P117=8.308886765309103e-05, P118=0.00010651539809045686, P119=0.00012987579765594184, P120=0.0002178499955384791, P121=9.444715897253986e-05, P122=5.124341876547807e-05, P123=5.643481004520755e-05, P124=7.771061932572842e-05, P125=6.959913069592623e-05, P126=6.880088428731188e-05, P127=4.192660247627292e-05, P128=5.7432677662421676e-05, P129=6.00451994622181e-05, P130=4.004651106645669e-05, P131=2.4915075332101425e-05, P132=1.1770599740621397e-05, P133=1.1122958244383856e-05, P134=1.3784400619658794e-05, P135=8.250908999567285e-06, P136=1.2836283642056452e-05, P137=1.5217009157175376e-05, P138=1.9096111925827698e-05, P139=1.750586075999789e-05, P140=1.4851383295362178e-05, P141=1.6730665622629816e-05, P142=1.5080716812548076e-05, P143=8.53830972678164e-06, P144=3.497117387110076e-06, P145=2.4809453095227898e-06, P146=1.6936469430054364e-06, P147=1.8727457775035001e-06, P148=1.5005292571628285e-06, P149=8.615872428425022e-07, P150=4.90710227748412e-07, P151=2.172928972628151e-06, P152=2.1837424486736667e-06, P153=2.6502519623249696e-06, P154=6.4060580504133805e-06, P155=1.3836141216145939e-05, P156=1.2809327148324166e-05, P157=8.023452599136118e-06, P158=8.785052282863058e-06, P159=6.830493172159772e-06, P160=2.2135559319172712e-06, P161=2.3190348664489374e-06, P162=3.5245301795535663e-06, P163=3.1670111484690815e-06, P164=5.26680466097055e-06, P165=7.010710503995017e-06, P166=6.150817334228192e-06, P167=3.4922649340133756e-06, P168=1.701756442698502e-06, P169=2.1403936607041535e-06, P170=1.3837943150477163e-06, P171=1.521805060028316e-06, P172=1.137696345585889e-06, P173=1.2414765684244357e-06, P174=2.105734735841977e-06, P175=2.0326102675059987e-06, P176=1.4496827856651953e-06, P177=7.873604338497894e-07, P178=1.8276403075109684e-07, P179=1.3414129085033123e-08),Row( P0=-776.06946429526, P1=75.61867843240813, P2=2.53822778865774, P3=14.421876407134805, P4=-8.863403124027808, P5=-8.135872713649432, P6=-2.3320994413365526, P7=0.5380394280607845, P8=0.9435679582218108, P9=-1.3544436314466048, P10=-2.8104385022857463, P11=-4.121213823076182, P12=-3.6058263147153955, P13=-5.976686082863788, P14=-1.4534512713790761, P15=-3.4440356040040956, P16=-3.153709676315333, P17=1.6287195629334519, P18=-2.6158752718211837, P19=-2.9946181599486694, P20=0.6899146718368012, P21=-2.3353766106355804, P22=-4.023008867541145, P23=-1.2589138170650112, P24=-3.5461709035000286, P25=-2.135081029294109, P26=-5.5050119865053375, P27=1.294361699846448, P28=-3.9279551497001663, P29=-1.5746493262066787, P30=-3.2813018636636215, P31=0.18471260750206545, P32=-4.7440022193234475, P33=0.22075325063649331, P34=-3.2222641342674647, P35=-1.4835701504178351, P36=-2.5178419042070352, P37=0.07655158569093118, P38=-0.5295819959673467, P39=2.301423367122413, P40=0.7218115283197228, P41=0.7203990355123279, P42=0.7238658834933537, P43=0.763711136591717, P44=0.7871330965188222, P45=0.7375493731615391, P46=0.7207127699236318, P47=0.7168115572714552, P48=0.7228373768507048, P49=0.7177426255095819, P50=0.7180999979385554, P51=0.7218097311637182, P52=5.280838502579787e-06, P53=1.568833348621864e-06, P54=3.218764695446867e-06, P55=0.00011664816717620407, P56=0.0009071193460240045, P57=0.0021623416511615673, P58=0.0009971584370449304, P59=0.0001512680637249034, P60=0.00014015694358214493, P61=0.0006516208712938616, P62=0.0013448570267808582, P63=0.001923042025644042, P64=0.0011796655322321146, P65=0.0008874862704794692, P66=0.0006881825570231831, P67=0.0005966036744206104, P68=0.00036499878120333965, P69=0.0006522586999116763, P70=0.0013659631150310728, P71=0.0011077252678525714, P72=0.0006023237988949959, P73=0.0003433833957686523, P74=0.00021854286058511295, P75=0.00031388144774209786, P76=0.0008855802462119343, P77=0.002780411524969978, P78=0.0036794841021595306, P79=0.0030028629825870983, P80=0.0011101752627457196, P81=0.0010478176881511387, P82=0.0011081879496945399, P83=0.001220338432062427, P84=0.0019568637747605236, P85=0.0016226221230711039, P86=0.0007129998279489149, P87=0.0005125225917798709, P88=0.0011877840400900067, P89=0.001963595185806735, P90=0.0017230917817449654, P91=0.0010041228007027976, P92=0.00029917795056825234, P93=0.0003742525787966511, P94=0.00034742828922143694, P95=0.00011503390501492123, P96=3.2055414385513896e-05, P97=9.261635723999164e-05, P98=0.00012681171580244128, P99=5.8531058891568345e-05, P100=3.1858578701493056e-05, P101=4.187637805100421e-05, P102=4.396053706159107e-05, P103=4.1755431205989856e-05, P104=2.6174324521508238e-05, P105=2.4972856214918275e-05, P106=2.7115214775894644e-05, P107=3.20642373514029e-05, P108=2.9474231559144827e-05, P109=2.7639112406410298e-05, P110=3.921753562043445e-05, P111=0.00012006668779301777, P112=0.00012446431615997748, P113=8.563003921016882e-05, P114=4.368799781219973e-05, P115=3.316311786114836e-05, P116=2.026722605353394e-05, P117=3.210306452174417e-05, P118=3.056049622195358e-05, P119=1.2362336258001298e-05, P120=2.4355601345913413e-05, P121=1.6324530120832714e-05, P122=1.265755045681204e-05, P123=2.3416497806698956e-05, P124=4.4799671127441695e-05, P125=8.870891302234827e-05, P126=3.664116634814828e-05, P127=3.710810740780528e-05, P128=2.9152759677601795e-05, P129=1.5669858301483003e-05, P130=7.79385704847007e-06, P131=2.4789795216235637e-06, P132=2.3511871347482365e-06, P133=1.8760922047947717e-06, P134=2.608302897007974e-06, P135=5.28678827771526e-06, P136=4.745027415294947e-06, P137=9.767822486377997e-06, P138=1.549053546164239e-05, P139=1.3379347892920713e-05, P140=1.5002081416665823e-05, P141=1.5795652838220034e-05, P142=1.8615702313465546e-05, P143=2.4379293081777802e-05, P144=2.0875482046059444e-05, P145=1.6966535723990597e-05, P146=8.123649271704568e-06, P147=1.2049575429633475e-05, P148=2.4807278607142468e-05, P149=7.509179918133433e-05, P150=8.314578499010881e-05, P151=3.749586694625345e-05, P152=6.0752741214370416e-05, P153=9.099939937100271e-05, P154=6.71183307680146e-05, P155=4.3273125595339715e-05, P156=2.987396027445697e-05, P157=2.562850863024203e-05, P158=1.9753240931263427e-05, P159=1.2642284978359946e-05, P160=1.0778818004887058e-05, P161=1.0616271362467425e-05, P162=3.6636371667493284e-06, P163=2.4521847814875375e-06, P164=3.065392784267109e-06, P165=5.157661407822429e-06, P166=7.407150027457854e-06, P167=5.7301244175779775e-06, P168=4.791728583726833e-06, P169=1.1964234262792906e-06, P170=3.7454539417737875e-07, P171=2.867696862847995e-07, P172=1.1311290983512656e-06, P173=2.945523272244605e-06, P174=6.807161065223107e-06, P175=1.268780291176139e-05, P176=2.0605955645744162e-05, P177=1.4664235262918168e-05, P178=5.489877317505758e-06, P179=3.0034989415680586e-07),Row( P0=-583.0607545688143, P1=51.152101410107484, P2=-6.116995224394131, P3=4.391952795691128, P4=-11.680342072608209, P5=-16.49062825412683, P6=-4.697860471352149, P7=-3.0177291278744787, P8=-6.844827045899876, P9=3.102148098936532, P10=-5.275440827840858, P11=-3.919255112583377, P12=-5.693206013776281, P13=-9.61358008785416, P14=-3.0439824408344016, P15=-0.627715163139634, P16=-5.284768784551284, P17=1.4544288566145278, P18=1.3184109847435561, P19=3.3427841535019907, P20=5.713005886736765, P21=2.6606257240392024, P22=4.001238762644279, P23=9.253931972671054, P24=1.8367722697188567, P25=4.524448083405674, P26=3.9139414021238053, P27=7.740366075189796, P28=0.21687404201233096, P29=4.860810303122552, P30=2.306890581878842, P31=2.6584677661737164, P32=-0.284681658056041, P33=2.0151326883220992, P34=2.3744217564873322, P35=-1.0834946975467186, P36=0.25290646149223933, P37=0.5825178442376172, P38=0.3301965056089423, P39=4.57301786376613, P40=0.6269676926046083, P41=0.5964957793933015, P42=0.612176963077019, P43=0.6254570812786528, P44=0.6595820253209185, P45=0.676804813469637, P46=0.6542086237536952, P47=0.6794338970150607, P48=0.7446835393639651, P49=0.7391435128650965, P50=0.6662754840385506, P51=0.6049408094072513, P52=0.00020869058073816514, P53=7.111141554200668e-05, P54=1.4560357204239982e-05, P55=9.779874919005721e-06, P56=2.6094809267946714e-05, P57=0.00016304993677562296, P58=0.0004891307089887071, P59=0.002979675787184559, P60=0.02154903527918286, P61=0.08076589857748623, P62=0.15765095745449134, P63=0.15730245974583476, P64=0.1194309752965737, P65=0.08057782207991279, P66=0.03912331153964998, P67=0.009634785654240637, P68=0.0028372866659183865, P69=0.004841185988834168, P70=0.009966464223236993, P71=0.018340439694997483, P72=0.0461349741128536, P73=0.048612245491912745, P74=0.03973119526328797, P75=0.08260472657158223, P76=0.22071756077906796, P77=0.7338225925539091, P78=0.5907317065414954, P79=0.24408621667979954, P80=0.1601323673185969, P81=0.10660232305909383, P82=0.05266962272117952, P83=0.06610018174168251, P84=0.08515370166779211, P85=0.05756307881085072, P86=0.062289841834938464, P87=0.07298391513969447, P88=0.11618829802735822, P89=0.13350911927920997, P90=0.06742229162275186, P91=0.0211583684586837, P92=0.03912843118085023, P93=0.03918275620054154, P94=0.017807847584496023, P95=0.0074031719135390864, P96=0.0032271968173749593, P97=0.011778447334800986, P98=0.022620070126596274, P99=0.013150507737780504, P100=0.0061057102891977945, P101=0.010647690101691467, P102=0.007988458002086538, P103=0.007459092529675569, P104=0.047369067134516524, P105=0.1015743051694779, P106=0.021546807592835414, P107=0.00799356051694844, P108=0.014502761123622416, P109=0.023555129352673326, P110=0.005831105823610681, P111=0.009232133787195923, P112=0.003573841106323391, P113=0.00048219644653199313, P114=0.00036543722109387626, P115=0.0003000592934260222, P116=0.0003903184413597331, P117=0.0004978717897069306, P118=0.0008185692604247614, P119=0.0013694289795859401, P120=0.0011386136755315776, P121=0.0007657268093836703, P122=0.0024629034949446146, P123=0.000844886324803848, P124=0.00033077636231272, P125=0.0017602659203444955, P126=0.0014983086850754999, P127=0.0018613288322129147, P128=0.011785706382267096, P129=0.01285520042409744, P130=0.006074972551757561, P131=0.0013029849293585243, P132=0.0005827949107205794, P133=0.0010695639449156168, P134=0.0014017467951529057, P135=0.002958941864653437, P136=0.006041421647584434, P137=0.008265332039885442, P138=0.008226457670407717, P139=0.003552053652462778, P140=0.003947547748136183, P141=0.0022908661995272076, P142=0.0025239534657580873, P143=0.0015297598781000505, P144=0.0005465771977356128, P145=0.00034306650175673783, P146=0.00023940696121832066, P147=7.620438901806355e-05, P148=4.862866957560918e-05, P149=7.715080980079727e-05, P150=0.00025032072527262966, P151=0.0006872730415698815, P152=0.0005886018832262858, P153=0.00035906142191754055, P154=0.00046380721931943447, P155=0.0005061921787776104, P156=0.0003924703224965735, P157=0.00021039216180492618, P158=0.0001698806033185976, P159=9.739531360146931e-05, P160=5.4689288219066754e-05, P161=3.403964973284808e-05, P162=5.436204519433483e-05, P163=0.00010938453432064965, P164=0.00011471482415709077, P165=0.00013166180615765486, P166=0.0001447027108506664, P167=8.398945117799159e-05, P168=4.9934723754195796e-05, P169=2.903196959892077e-05, P170=2.0842387249195284e-05, P171=1.9451359444156203e-05, P172=3.198875989790594e-05, P173=4.796311681007405e-05, P174=7.017355626229823e-05, P175=7.844335606925536e-05, P176=5.985870539868594e-05, P177=3.1609797036444625e-05, P178=1.5124822658014423e-05, P179=1.5989534834664644e-06),Row( P0=-665.5943274107648, P1=64.79163872029757, P2=-11.591663457172137, P3=8.992386515679952, P4=-8.69809081237688, P5=-16.419775226058135, P6=-5.594132243281195, P7=0.7470931890626367, P8=2.3497009080159614, P9=-5.627532501284125, P10=-7.525307811725418, P11=0.7513460756208946, P12=-8.338876778643677, P13=-5.890484661831944, P14=-0.7017587855364267, P15=-3.645487643556593, P16=-2.2311102976461816, P17=0.6718896930269104, P18=-4.542988052772973, P19=-0.4412707757340619, P20=-1.8054911905849018, P21=-0.7684630676917127, P22=-4.563342281396937, P23=0.09281279596366687, P24=-3.9493548275798016, P25=-3.6901993784206564, P26=-2.4894438557085143, P27=-0.32449300482731747, P28=-2.2610623344792606, P29=3.8492596112640967, P30=4.364546358526921, P31=8.229052990911677, P32=6.00029416457378, P33=8.74041364039167, P34=6.107482988675298, P35=4.961883700803052, P36=3.4034319932618047, P37=4.004482407984375, P38=1.8689566701029507, P39=3.531910429443188, P40=0.7931536440977864, P41=0.7528635091988968, P42=0.6981772209103495, P43=0.6464236282255109, P44=0.5985096111314795, P45=0.5696878904322834, P46=0.6010562658982238, P47=0.6358176354199973, P48=0.6719593411911852, P49=0.703706579992637, P50=0.7191931593743653, P51=0.7553136932749701, P52=4.1222129189974005e-05, P53=1.4212447740801299e-05, P54=2.418988011801987e-06, P55=8.389624510588675e-06, P56=9.294880798840109e-05, P57=0.0009432295165668381, P58=0.006732515009064141, P59=0.017787736338584558, P60=0.010264196391555996, P61=0.0015388222496181896, P62=0.00024790340844122366, P63=0.00034765205095376903, P64=0.0010141757040251884, P65=0.006129197087972122, P66=0.020585413788033732, P67=0.031165711957804845, P68=0.02319054039995174, P69=0.013996338023625716, P70=0.005043491813272205, P71=0.0009566674714241755, P72=0.0005616998746923706, P73=0.0011733224528368959, P74=0.005507624104857247, P75=0.014763399630198827, P76=0.02195730277942411, P77=0.04600093486118826, P78=0.06914866477877539, P79=0.04359175210468893, P80=0.007149469826004539, P81=0.00429893946308422, P82=0.011422642417826017, P83=0.02213309928299329, P84=0.02383413449678383, P85=0.014649251391476327, P86=0.009729144725647893, P87=0.007146118390475413, P88=0.012417783846584313, P89=0.031170230326423413, P90=0.048289462006248646, P91=0.015694371226642207, P92=0.0015823249217968897, P93=0.0013024800408660655, P94=0.0024626166851009702, P95=0.0021406643114342374, P96=0.0009770083998320912, P97=0.00035834742058238073, P98=0.0002496465031263054, P99=0.000850051545696023, P100=0.0008154001952986174, P101=0.000430917812575215, P102=0.0002521564612664983, P103=0.0005002935774790829, P104=0.0006728517678064526, P105=0.0004084141863360479, P106=0.0002208195508847329, P107=0.0006668053763270187, P108=0.00020085886431393663, P109=0.00022616245637713353, P110=0.002364412730757958, P111=0.00282628695894292, P112=0.00023892461959697077, P113=0.00029930389019574053, P114=0.0018559946344755677, P115=0.002474636896029719, P116=0.0006972340405838319, P117=0.0006926970271478634, P118=0.0011151992384775546, P119=0.0006399080999866734, P120=0.001460498181406939, P121=0.0004980111125761006, P122=0.0010979870927602493, P123=0.0005616170402822665, P124=0.0010194641327096836, P125=0.0007894842709043236, P126=0.0009550696733384277, P127=0.001395884969255852, P128=0.0013481292552036943, P129=0.0007055433368669604, P130=0.00040027192180588217, P131=0.00022148657023753543, P132=0.00011691420273615423, P133=5.439095348312715e-05, P134=4.2316096493588435e-05, P135=3.471969556155604e-05, P136=3.1704550560811723e-05, P137=4.669469854339257e-05, P138=7.089883898884422e-05, P139=0.0001434142525092745, P140=0.0001880456552815197, P141=0.0003872358306344627, P142=0.00027330320713964563, P143=0.00017516456717972458, P144=0.00010986027776686098, P145=7.22911908233075e-05, P146=3.9038978476461735e-05, P147=4.8856616506711656e-05, P148=7.990011164085035e-05, P149=0.00021497581486718426, P150=0.00042179581263834034, P151=0.0004968317634606772, P152=0.0003083305808683838, P153=0.00017313750144885108, P154=0.00012959498997252653, P155=0.0001132661485528227, P156=0.00029103709118151846, P157=0.00022466133279560438, P158=0.0001908477867624403, P159=0.00015195699126225012, P160=0.00010565961472396384, P161=5.2036717892805445e-05, P162=3.717999861543263e-05, P163=5.4295965435436844e-05, P164=6.513958499203204e-05, P165=5.949649321054395e-05, P166=4.213500944217907e-05, P167=6.203540274354981e-05, P168=5.4963125965432484e-05, P169=3.379569260045649e-05, P170=3.281469628446621e-05, P171=3.2676262410276244e-05, P172=2.1684547406077228e-05, P173=2.5441176060114593e-05, P174=1.8598139615046917e-05, P175=1.9280191953359913e-05, P176=1.8336139540951692e-05, P177=8.234876734209928e-06, P178=2.0049599035640572e-06, P179=1.8029173167437636e-07),Row( P0=-499.39573203089515, P1=71.8243477574077, P2=-36.58243736214159, P3=-4.6788934824752575, P4=-19.82287191815054, P5=-29.235255605706808, P6=-2.5482030571573597, P7=-1.837282529192782, P8=-8.78325458769571, P9=-2.3373670734897734, P10=-16.646144200143716, P11=-6.940431994825919, P12=-5.636357748812518, P13=-5.8896570154372965, P14=4.171849767787606, P15=11.09899289869461, P16=8.80737966948171, P17=14.99634144367943, P18=4.337541961962585, P19=6.056812940173086, P20=2.2505063604507654, P21=0.2335468128365815, P22=5.639341631440897, P23=7.886903041795218, P24=-1.273529931471195, P25=-1.6247626945702547, P26=-0.8107269134866235, P27=1.0499051505549208, P28=3.2870564983366837, P29=6.880324673365064, P30=0.967979368655268, P31=4.887710123951335, P32=3.069800619737301, P33=5.542082685695106, P34=0.9765370010034337, P35=-1.6742882944007738, P36=-1.632598702592179, P37=1.3851783858097861, P38=4.232070697149107, P39=3.2824634008941405, P40=0.6934566738738255, P41=0.702970638421782, P42=0.5816637495737779, P43=0.5275507556908573, P44=0.5403273368333863, P45=0.5182806603307103, P46=0.5027091474703399, P47=0.5222836767995066, P48=0.5970546867674271, P49=0.5910055867106984, P50=0.5703731136445161, P51=0.591630320943788, P52=0.00035198627767568967, P53=0.00013052034787401645, P54=3.587677922097587e-05, P55=1.727012781233442e-05, P56=4.490649855127096e-05, P57=0.00012540744081278028, P58=0.0002162291266871069, P59=0.00048746518816010715, P60=0.0013349427190743138, P61=0.003427572716031028, P62=0.009276486720191147, P63=0.01617640346386708, P64=0.023174231913480135, P65=0.06286579760166304, P66=0.10393236080084568, P67=0.12165736599824215, P68=0.17072290986310762, P69=0.169732305345346, P70=0.08361953517404173, P71=0.05120417146924405, P72=0.013890693393925077, P73=0.005060272717705424, P74=0.0042031873206034446, P75=0.005416933810081152, P76=0.008247765200707819, P77=0.021954958100775735, P78=0.0371728759213468, P79=0.055327305014132175, P80=0.05404678918403161, P81=0.07935431361747966, P82=0.17076258427244875, P83=0.4894132024833345, P84=1.5531422523477179, P85=1.7586745187847075, P86=0.9873786636769621, P87=0.6854783398386588, P88=0.242880428408344, P89=0.16750318032986652, P90=0.28106650461251353, P91=0.27577533879667526, P92=0.25574203613622376, P93=0.2466966092688852, P94=0.3931918057706357, P95=0.3521675637888661, P96=0.26642393726192914, P97=0.43405008499526293, P98=0.620233928310916, P99=0.7948108603320525, P100=0.08575004371159087, P101=0.005917006637301038, P102=0.005081999283839497, P103=0.008880180610437327, P104=0.009936054408111959, P105=0.03915683258969032, P106=0.11072059272965731, P107=0.231297159562453, P108=0.08430997852848922, P109=0.013566116614205878, P110=0.023647346461125287, P111=0.01652819343703479, P112=0.010588686370845236, P113=0.010911655058317876, P114=0.005472791795187694, P115=0.0052255183804370735, P116=0.004106116711525634, P117=0.0069479130066088255, P118=0.023584358820734123, P119=0.019953066463098072, P120=0.003272817490181675, P121=0.0037328106594601445, P122=0.007468632894287698, P123=0.005410615254246342, P124=0.0023050792056015933, P125=0.001793304685180532, P126=0.0028822382228967545, P127=0.004779550775473858, P128=0.002995452913078603, P129=0.0027696488941893263, P130=0.004741278965360686, P131=0.008945255075537142, P132=0.011350712000108943, P133=0.007918869219142352, P134=0.002471552716466409, P135=0.002320936427395829, P136=0.0021966194394043054, P137=0.0021673777419621808, P138=0.0030708549740752055, P139=0.004213869163653223, P140=0.003303570460455041, P141=0.003207796401971456, P142=0.0026640840028252055, P143=0.001170939091854633, P144=0.0006258441306346876, P145=0.00018261524871666715, P146=0.0001624759205288646, P147=5.534477758079022e-05, P148=5.135619596552576e-05, P149=2.7726041372135324e-05, P150=3.5073604313401e-05, P151=4.508897853589336e-05, P152=3.8951688709451685e-05, P153=4.600832882753262e-05, P154=4.0101093258414975e-05, P155=3.232566255912834e-05, P156=9.658754012340642e-05, P157=6.701296289790648e-05, P158=6.186717003644408e-05, P159=6.277071083092316e-05, P160=4.926915442642981e-05, P161=5.3255690139231994e-05, P162=5.6717866823817206e-05, P163=8.901416508961358e-05, P164=6.305606412444738e-05, P165=5.8265049699979746e-05, P166=5.701784030668613e-05, P167=5.8694715051485874e-05, P168=5.7253127695388005e-05, P169=4.191277288844384e-05, P170=5.478093771220556e-05, P171=4.602773380335629e-05, P172=5.2144278748241565e-05, P173=4.370608885102936e-05, P174=3.6246501069819776e-05, P175=3.0627029338781265e-05, P176=2.4134430475758802e-05, P177=1.1841074482990295e-05, P178=4.412170479446851e-06, P179=4.609737636321809e-07),Row( P0=-762.053957071183, P1=80.77961969473158, P2=13.162316769835142, P3=15.550579836859121, P4=0.26027213125661525, P5=-2.42814612579318, P6=-0.11509124011123288, P7=7.7760487135134815, P8=3.946427103056421, P9=-2.055102624853798, P10=-0.6168619545922867, P11=3.5085167654469083, P12=-5.509821826208851, P13=2.1403674568117648, P14=-3.2143933732940866, P15=-0.2287678981832604, P16=0.6764554674863597, P17=-0.4890078050550797, P18=-0.8638178059264324, P19=-2.796049246480835, P20=-2.6519071123529128, P21=-0.0034532243454965038, P22=-7.141939084127027, P23=2.1058420682238204, P24=-4.69209323446391, P25=-4.796571621764877, P26=-4.694560303174901, P27=-0.250404319941009, P28=-6.459012155445481, P29=-1.844359796356779, P30=-3.9881707659555024, P31=-1.6822861950688142, P32=-3.0458269371407987, P33=-1.9837789500032044, P34=-1.21170986766578, P35=1.0982450083265436, P36=1.2093770019820027, P37=5.454221688168369, P38=4.355180452257847, P39=6.1802694339991415, P40=0.6815167801048856, P41=0.7478875286292219, P42=0.7178671048996975, P43=0.701296851609011, P44=0.7374472516252532, P45=0.7158592292767095, P46=0.6921303178714355, P47=0.7057639574640551, P48=0.7658446301347602, P49=0.776132284634542, P50=0.7292769949896327, P51=0.6606188183978966, P52=3.365227416746036e-05, P53=1.6768665698671713e-05, P54=6.863188357877063e-06, P55=0.00028426027750010904, P56=0.0025966302837605922, P57=0.008382340835800152, P58=0.006001183203231711, P59=0.001291383976421816, P60=8.828961062004924e-05, P61=7.463221433378488e-05, P62=0.00020405374831372082, P63=0.0007833390562537909, P64=0.0020139790445560723, P65=0.005897709072963131, P66=0.003352290498170634, P67=0.0002593343595241381, P68=0.00021161361778981636, P69=0.0004604489506694579, P70=0.0006716399553047474, P71=0.0009193088298326036, P72=0.0014558091087967413, P73=0.001387315034085303, P74=0.0003723790823006718, P75=0.00012127643047057883, P76=0.0001411786692603998, P77=0.0006968248330324789, P78=0.002135005541232436, P79=0.003545796003404479, P80=0.0026189957768474885, P81=0.0005972825848530351, P82=0.0002683493984325832, P83=0.00047477747524609894, P84=0.0012412072638186568, P85=0.0022967773880459637, P86=0.0046427928945230795, P87=0.003012121518140941, P88=0.00014752649474106285, P89=5.823548884125595e-05, P90=0.00017840507645361245, P91=0.0006683353730422946, P92=0.002341160214352463, P93=0.00033657899467763987, P94=3.9432569522103405e-05, P95=7.216711566285782e-05, P96=0.00018692635336175928, P97=0.00034193272860562676, P98=3.212948443570714e-05, P99=2.9566789805082864e-05, P100=8.799542458972446e-05, P101=0.00013801416219812732, P102=1.590691763002767e-05, P103=1.2104211284514117e-05, P104=5.9524596990971436e-05, P105=3.698305955634017e-05, P106=2.951728833320737e-05, P107=0.0001257439120845748, P108=0.00011315908174924492, P109=1.7722901469917147e-05, P110=0.00021551291498198808, P111=0.00040663523679392445, P112=3.820585214825104e-05, P113=1.6357108772177066e-05, P114=1.582769048219487e-05, P115=9.718840125644512e-06, P116=2.6225672284577414e-05, P117=1.1463680236565903e-05, P118=1.2973809643870513e-05, P119=8.830934328803472e-06, P120=1.3761386119530979e-05, P121=6.920937051051083e-06, P122=1.7785401130131552e-05, P123=9.972424745865404e-06, P124=3.123332376333468e-05, P125=2.5420423912994616e-05, P126=4.478193265369859e-05, P127=2.8220345019352678e-05, P128=2.5850653397775116e-05, P129=1.9972013773558518e-05, P130=5.072104111688794e-06, P131=2.7484452254461156e-06, P132=2.9205623819267102e-06, P133=1.1270499203696147e-06, P134=9.646515111919574e-07, P135=1.2494813262489683e-06, P136=1.2609972345617641e-06, P137=1.7570956561371827e-06, P138=3.622646092375713e-06, P139=3.3087043175088704e-06, P140=4.3751736564754584e-06, P141=3.5657632729090514e-06, P142=3.2160986030309723e-06, P143=4.281754131126052e-06, P144=2.641894140757776e-06, P145=3.773972547995627e-06, P146=6.469218328953422e-06, P147=1.450870403674978e-05, P148=4.342736470078355e-05, P149=8.966303173751963e-05, P150=0.0002540038955280786, P151=0.00015280215851181108, P152=2.9173884671451167e-05, P153=1.5970819485928123e-05, P154=1.3201186431387565e-05, P155=1.276941180915271e-05, P156=9.014541422229274e-06, P157=4.8068804066256625e-06, P158=3.2382433960357097e-06, P159=2.210909988276634e-06, P160=2.1074923941454664e-06, P161=2.909397827689158e-06, P162=5.949672811584036e-06, P163=5.6010667631794264e-06, P164=3.789711743863219e-06, P165=3.828448844154115e-06, P166=3.207341813318145e-06, P167=3.476407840487901e-06, P168=2.0518507654950495e-06, P169=2.2174290828334033e-06, P170=2.9999193537800226e-06, P171=5.228498396197383e-06, P172=5.423199695161856e-06, P173=4.590207670451101e-06, P174=4.096406969041682e-06, P175=3.5166188600804505e-06, P176=2.425780744664804e-06, P177=1.1838879716752836e-06, P178=4.2307387143130907e-07, P179=4.609369737637913e-08),Row( P0=-819.246225146041, P1=66.30429873715242, P2=8.0932934429695, P3=15.703644169694893, P4=-0.10642213317836471, P5=-4.039582119452251, P6=0.5461503421213033, P7=3.9548724749485324, P8=0.6705193683136483, P9=-3.2420584548970477, P10=0.4385986908514369, P11=-1.0001863181655417, P12=-5.0858389360809495, P13=-1.0877887212549897, P14=-6.44969387598333, P15=-5.257662123554159, P16=-0.18431149494032206, P17=-3.910256552657442, P18=-3.0321352791057707, P19=-2.4531041161914047, P20=-3.76705781568291, P21=-3.46008459286477, P22=-5.143156864775879, P23=0.23703968760998342, P24=-3.84634856816445, P25=-2.5234236842456315, P26=-2.903491344287493, P27=-1.0334390800401787, P28=-4.764319138710543, P29=-1.9920146783467638, P30=-3.705302681931342, P31=-0.4270690216855477, P32=-3.5627393853018594, P33=-0.8477726484966585, P34=-4.063198922329412, P35=0.14985924984350354, P36=-3.44852988452287, P37=0.5112571446130473, P38=-1.540816540951175, P39=0.8758822522169467, P40=0.7544592547257312, P41=0.755505763734209, P42=0.7648987363806787, P43=0.8128380113322619, P44=0.8248884248438185, P45=0.8080817101148181, P46=0.7649809950844547, P47=0.7374376090604201, P48=0.7285242965455035, P49=0.7083972376139038, P50=0.7063870636821975, P51=0.7370166419340274, P52=4.819075212806935e-06, P53=2.086472069471687e-06, P54=8.316896496788546e-06, P55=0.00017525327465441263, P56=0.002094815331509315, P57=0.0038337582330139354, P58=0.0012555854211005206, P59=0.0004099361634236705, P60=0.0001404405461536167, P61=0.00037739329322432916, P62=0.0007414351801800363, P63=0.0005248445959059862, P64=0.00014693383168510473, P65=0.00016404745275396608, P66=0.00017256176489196523, P67=0.00014700528577373017, P68=0.00016237679739728638, P69=0.0001257333277064078, P70=0.00011311272418599462, P71=0.00011399351752824918, P72=0.00020206787001954632, P73=0.00020081800598599074, P74=0.00011077850223326282, P75=7.387112170218152e-05, P76=5.921985218939152e-05, P77=0.00011075432835052081, P78=0.00016864262334510306, P79=0.00027465215693892953, P80=0.0002995149844319697, P81=0.0001951519955216088, P82=0.00010100415890641333, P83=0.0001055808202356845, P84=0.00015917222787546864, P85=0.0001020151252854862, P86=7.89470336724168e-05, P87=5.5640663350709644e-05, P88=4.4002612691597784e-05, P89=4.887988045292272e-05, P90=6.892634900984653e-05, P91=8.13327426445598e-05, P92=8.938095864319879e-05, P93=3.359374089186204e-05, P94=5.439734043939404e-05, P95=5.030928811615953e-05, P96=1.4068221483091609e-05, P97=1.881770964397886e-05, P98=2.878374236515394e-05, P99=1.9421464538526216e-05, P100=9.283190675801832e-06, P101=8.852971078592642e-06, P102=5.812059370509788e-06, P103=5.787775315114582e-06, P104=6.0841210434993625e-06, P105=6.473286468902001e-06, P106=1.0243875213871296e-05, P107=1.0817980010386458e-05, P108=8.614006195838062e-06, P109=6.554481731390359e-06, P110=1.4432329105039866e-05, P111=2.1571582709578134e-05, P112=7.351862775476014e-06, P113=4.005137464579487e-06, P114=6.6621592645601005e-06, P115=1.3758565065839256e-05, P116=7.242738047026341e-06, P117=3.991483753381512e-06, P118=5.5713745315199204e-06, P119=8.792047603804702e-06, P120=6.6219711206783655e-06, P121=3.586091380641309e-06, P122=4.503332840913444e-06, P123=3.0281189943521852e-06, P124=4.613806942508376e-06, P125=8.452372496742996e-06, P126=1.5970850445933278e-05, P127=1.8827127038840515e-05, P128=1.0223931591823921e-05, P129=1.3366223239588164e-05, P130=1.6770819282060373e-05, P131=3.879172388002225e-05, P132=3.529650634580296e-05, P133=2.7155169314697215e-05, P134=1.7854424543412228e-05, P135=9.922172549874131e-06, P136=1.1573675595272743e-05, P137=3.815916640236563e-06, P138=4.665680439262464e-06, P139=4.271745632829056e-06, P140=2.113271209898676e-06, P141=1.7191530988012937e-06, P142=2.428919708459976e-06, P143=2.8843028824938835e-06, P144=1.8638056050757844e-06, P145=3.388308574596773e-06, P146=8.134239478531752e-06, P147=1.1456651188767895e-05, P148=1.8133378630256303e-05, P149=3.509969091527236e-05, P150=3.580916929474706e-05, P151=1.2086853835285196e-05, P152=1.751972191554507e-05, P153=6.6391532853211675e-06, P154=2.917381336113847e-06, P155=1.8661563641926493e-06, P156=1.2153508979184695e-06, P157=9.050801765740644e-07, P158=1.5342255494652558e-06, P159=1.2125301521422593e-06, P160=1.9589229960699967e-06, P161=1.6097539881295903e-06, P162=1.4186734827998407e-06, P163=1.576216044420378e-06, P164=1.3458387658765368e-06, P165=4.682323008332788e-07, P166=3.098044134607294e-07, P167=3.543808910317184e-07, P168=3.7705864691250883e-07, P169=6.239067991370121e-07, P170=4.365101541386742e-07, P171=2.1692022541988274e-07, P172=1.3059621305880057e-07, P173=2.1468970682212173e-07, P174=4.4404476525096006e-07, P175=1.2281014901581737e-06, P176=2.384555070191825e-06, P177=3.570490069715831e-06, P178=1.4383802685124874e-06, P179=1.637468701425273e-07),Row( P0=-715.4026856678435, P1=75.91126661558488, P2=-7.192993983698772, P3=8.342698657747153, P4=-10.187702072848486, P5=-12.802561513229048, P6=-1.9429781773027612, P7=-6.954129148160822, P8=-4.607974013762099, P9=-2.72050991637523, P10=-0.8018000569802985, P11=-5.007405919145245, P12=-5.018319579461303, P13=-3.6109350218834324, P14=-2.4676220169237455, P15=-7.115387166962554, P16=-1.2231447360781045, P17=-0.2541609030677184, P18=-5.6369697870105995, P19=-1.0387524471807446, P20=-2.3245276728920405, P21=-2.456614488682566, P22=-2.7748135537112644, P23=-3.060076965205962, P24=-3.2832838543367777, P25=-2.278516669906237, P26=-5.103738206034044, P27=-1.190177112846355, P28=-3.427333275029132, P29=-3.7927629035384527, P30=-2.8149557867944055, P31=-0.8292024903000671, P32=-2.3289440190006605, P33=0.016309995656535903, P34=-2.7038464816138204, P35=-0.4488683505375832, P36=-1.0429779584650711, P37=1.7910777478914715, P38=1.150870327120309, P39=3.624097186649666, P40=0.7241992827554966, P41=0.7110383838171178, P42=0.7085987163148337, P43=0.7162121355612051, P44=0.747517661190064, P45=0.7718348212068008, P46=0.7643557350275368, P47=0.7391655804983975, P48=0.7651518947383824, P49=0.7593916697414228, P50=0.7177404289644927, P51=0.7105700798599183, P52=6.894598674780235e-06, P53=2.2494582568913827e-06, P54=3.237875178887466e-06, P55=5.3981327230526425e-05, P56=0.0006395537876125625, P57=0.0023626859899795235, P58=0.0015614170561832467, P59=0.0006135377726275324, P60=0.00017642203915226896, P61=0.0002591265521879195, P62=0.0012190938642617649, P63=0.0021079947018873466, P64=0.0023381254103101417, P65=0.0023568510694248683, P66=0.0018955736726246255, P67=0.002753030216234873, P68=0.001743384595988229, P69=0.0016945530218121262, P70=0.002518696300396408, P71=0.002802278777369313, P72=0.002064882934282582, P73=0.0015686686966669531, P74=0.0036002113927579844, P75=0.005980923352311228, P76=0.004413215692417642, P77=0.01077205292082169, P78=0.009625630200377981, P79=0.005525941097677564, P80=0.002940005637029609, P81=0.0027102401761226057, P82=0.003375868810987583, P83=0.0027940446177171212, P84=0.0019275393231761992, P85=0.001448969166636207, P86=0.0022326367731955303, P87=0.0019150698432374678, P88=0.0005278042591890201, P89=0.0003182636300767225, P90=0.00048293217659163334, P91=0.003665275532801025, P92=0.012763279728388017, P93=0.003367301946859103, P94=0.0005527734847043504, P95=0.0003048598812908625, P96=0.0007020590296636952, P97=0.0008788377846485587, P98=0.0009500006373281828, P99=0.000605655132611095, P100=0.00048709233887094547, P101=0.0007596785943794206, P102=0.0004062284045063409, P103=0.0002204653157762219, P104=0.0009804376136292921, P105=0.0005758582938742318, P106=0.00038550685732825063, P107=0.001276662123874605, P108=0.001644398781931765, P109=0.0005717271585050788, P110=0.00015402654952502236, P111=0.0002591963003978051, P112=0.0003165711176773062, P113=8.00657652658478e-05, P114=4.0497917142444044e-05, P115=4.942665317688918e-05, P116=7.566278511336479e-05, P117=3.8837185321945873e-05, P118=2.3393839083119783e-05, P119=1.9351210598478772e-05, P120=2.6730994300574753e-05, P121=2.3302227840558996e-05, P122=1.7939638310326333e-05, P123=3.845331779029786e-05, P124=6.753942170441145e-05, P125=0.00011004176409106971, P126=8.01242954642258e-05, P127=0.00016568623197136474, P128=0.00016198921340726592, P129=9.603364601445373e-05, P130=0.00010052935314455198, P131=5.293971196240067e-05, P132=3.6628871943946085e-05, P133=6.0093561308944924e-05, P134=5.050884710388719e-05, P135=5.011679694308986e-05, P136=2.929540556065279e-05, P137=3.616954639564155e-05, P138=2.3491529972506014e-05, P139=3.776513370927393e-05, P140=4.323533020230584e-05, P141=3.4121268959645755e-05, P142=3.376510605545913e-05, P143=2.5187018986873978e-05, P144=2.5787200271205875e-05, P145=2.0623016848252166e-05, P146=1.974223547620774e-05, P147=9.953314323027853e-06, P148=8.09554582215073e-06, P149=4.051846099952082e-06, P150=3.6841292120520304e-06, P151=3.911215207483599e-06, P152=1.289669441240579e-05, P153=1.9874379178257647e-05, P154=1.7386751881235498e-05, P155=2.4726575705098523e-05, P156=3.1599174328767215e-05, P157=3.829217145373264e-05, P158=2.1837229469500636e-05, P159=1.930651978678188e-05, P160=9.939064584133883e-06, P161=1.5878653364963013e-05, P162=9.869214833148902e-06, P163=3.843077343379123e-06, P164=1.1457479022450994e-06, P165=8.722425854164976e-07, P166=9.808519520426403e-07, P167=5.759706350964206e-07, P168=1.2000748086720783e-06, P169=1.6281217276425207e-06, P170=2.907249370897145e-06, P171=1.8675360331028565e-06, P172=1.8239537215424591e-06, P173=1.9391104326583533e-06, P174=2.5839839670267655e-06, P175=3.4553133075782773e-06, P176=2.0641464169939047e-06, P177=1.9993610698039274e-06, P178=1.8991118890149806e-06, P179=1.873904959803757e-07),Row( P0=-713.3478784827504, P1=62.90415462348277, P2=5.122807803508808, P3=19.279201091451906, P4=-4.612076492408492, P5=-10.9459473441527, P6=-0.6088413562097956, P7=1.5967818723263505, P8=-3.0617761770334737, P9=2.5067403997107722, P10=2.6368813173110164, P11=1.1727876737155272, P12=-1.827144115099967, P13=-5.437246026688326, P14=-6.07551506298324, P15=1.3536317920056535, P16=-2.7421229093739563, P17=-4.632054059900606, P18=-0.36309092101894846, P19=-0.19390945135575047, P20=-0.9009428702752912, P21=-2.7637506112524783, P22=-4.7506643334664505, P23=5.338648356784959, P24=-4.411887749632453, P25=0.4716343464321858, P26=0.1894546777623364, P27=6.021171947967425, P28=1.8205847509803066, P29=7.615636590873417, P30=4.851376621184557, P31=6.3463764869437185, P32=3.6243647195469206, P33=4.171491893110841, P34=3.34479912654739, P35=3.2268879718535044, P36=2.0841344482596558, P37=3.9007447547541183, P38=0.06862385273031155, P39=2.658002385441013, P40=0.7278766193237689, P41=0.7624390169843561, P42=0.7468287854180516, P43=0.7109305484728015, P44=0.663655373204016, P45=0.6652674192144571, P46=0.6651667159629446, P47=0.698339464260526, P48=0.7673310086918574, P49=0.7562089268716974, P50=0.6988221113813201, P51=0.6835023106469174, P52=0.0008417192631136955, P53=7.384677099833513e-05, P54=6.395427270577487e-06, P55=7.955221402091423e-06, P56=5.5749088818571914e-05, P57=0.00045536259322961225, P58=0.0024674705407104667, P59=0.012993491586146225, P60=0.02438804511248242, P61=0.016425676822079665, P62=0.004048064788897685, P63=0.0005850310781252704, P64=0.00014609148044513202, P65=0.000251854791898708, P66=0.0006160520964795159, P67=0.0012736880098886167, P68=0.003066498567522645, P69=0.00607716277578827, P70=0.008493945018604883, P71=0.0045113466940212696, P72=0.0015957924009877046, P73=0.0009648043364511503, P74=0.0005925129823770664, P75=0.0007039294922979653, P76=0.001144174021564724, P77=0.0034660357813488607, P78=0.004352660118297154, P79=0.0067473388349045755, P80=0.0069766159166641025, P81=0.005489103836390643, P82=0.003683733028212107, P83=0.001049748307154142, P84=0.0007670159785685644, P85=0.001491630610439462, P86=0.0014796334049369617, P87=0.0009161763516750929, P88=0.0004788986318976007, P89=0.00022467198220276098, P90=0.00015267050032849837, P91=9.82486282174221e-05, P92=0.00020379282339804747, P93=0.00019796365429860103, P94=0.0002875968305683172, P95=0.0003103872709137307, P96=0.00017755683051781342, P97=0.00022443570952063513, P98=0.00022945654619676474, P99=0.00025803472677170296, P100=0.000199392518953247, P101=0.00029707500612309004, P102=0.000425929299792256, P103=0.000339764482841224, P104=0.00033772764220444025, P105=0.0002922686522709271, P106=0.00020506962918459326, P107=0.00017983505872739829, P108=0.00011374783507893719, P109=2.7892641877239477e-05, P110=2.853885010929568e-05, P111=5.759476457215995e-05, P112=2.812566815589545e-05, P113=1.5917378059766615e-05, P114=6.017430906760724e-06, P115=2.2818818379548435e-05, P116=6.934772605592657e-05, P117=4.3795927762355395e-05, P118=3.284501053876085e-05, P119=3.869868773110109e-05, P120=6.41823130993191e-05, P121=3.0136575048187064e-05, P122=3.041135498499673e-05, P123=2.9839983036580116e-05, P124=2.03087088418363e-05, P125=3.853096046954013e-05, P126=2.58071365558545e-05, P127=4.0772158978995206e-05, P128=0.00010344957176586357, P129=0.000160813829241833, P130=0.00016714352450177146, P131=0.00019154231223998284, P132=0.00015498110734798892, P133=0.00027229151409950236, P134=0.00039439684002321547, P135=0.0005912674318866274, P136=0.0002489914113929731, P137=0.00015098512106348635, P138=0.00016297373883310612, P139=8.491547925472524e-05, P140=7.067429547014429e-05, P141=5.2107352266067486e-05, P142=2.5602397604973448e-05, P143=1.9571499408021125e-05, P144=1.4128866951284646e-05, P145=1.4676018742723886e-05, P146=1.5225403792104104e-05, P147=3.407968282402894e-05, P148=4.4454309473966796e-05, P149=5.2414706507371436e-05, P150=8.578662606650945e-05, P151=7.199843708666811e-05, P152=7.874416222192959e-05, P153=6.169418198227916e-05, P154=5.1657585707599625e-05, P155=4.1641524319764584e-05, P156=3.092033121671816e-05, P157=1.658242637534583e-05, P158=4.559923529926668e-06, P159=3.7291645342759524e-06, P160=5.637602202827427e-06, P161=1.0301288841208479e-05, P162=1.053824915609821e-05, P163=1.1784030377299439e-05, P164=1.7960195500022872e-05, P165=2.107325525320666e-05, P166=9.320143183494365e-06, P167=6.078555808353912e-06, P168=5.38737165548614e-06, P169=4.098837411700797e-06, P170=7.4978832070982994e-06, P171=7.756400116881267e-06, P172=6.866117525774631e-06, P173=1.0204094034670618e-05, P174=2.0196373870172737e-05, P175=2.271525921322921e-05, P176=1.8623574321241783e-05, P177=1.0369639245613202e-05, P178=3.608507847945282e-06, P179=3.0049890840089337e-07),Row( P0=-547.0011725342262, P1=57.23524923321443, P2=-8.78467560721619, P3=3.997402723850015, P4=-11.246698809926874, P5=-18.66156076573835, P6=-5.163297665375327, P7=-2.5108386708114314, P8=-4.898101119003432, P9=3.5196471952188246, P10=-4.7578175138605125, P11=-0.6575841230452836, P12=-3.95978179769461, P13=-12.547613086005367, P14=-2.5579593626048336, P15=-0.5867760084913406, P16=-4.425312172542382, P17=4.651540829336109, P18=2.848037974179128, P19=7.02854639568331, P20=10.872367109115434, P21=2.397897454377437, P22=6.278506786936096, P23=12.686545186183764, P24=2.2753425766853543, P25=6.584963506335995, P26=4.2775587963913315, P27=5.46026193639324, P28=-0.9964202560721577, P29=0.7871450923145634, P30=2.211509468305076, P31=2.721468009836131, P32=0.2820649259648603, P33=3.355273719385311, P34=2.634940610411501, P35=0.9796283495833931, P36=3.45027874332743, P37=2.711853979389181, P38=0.00835325531268885, P39=2.2195294538049364, P40=0.5502676814510005, P41=0.5389606360426618, P42=0.5216144862994601, P43=0.514243773225836, P44=0.5574694729056792, P45=0.638628793411318, P46=0.7287302687256202, P47=0.7399561649061368, P48=0.729927882611893, P49=0.7614120231888944, P50=0.6966603440719265, P51=0.5824332433264706, P52=0.0017500935552704584, P53=0.00026830832069637325, P54=1.7772790307712778e-05, P55=9.842225733442184e-06, P56=5.981400015591209e-05, P57=0.00026287919752148754, P58=0.00025177024734360633, P59=0.0008758423210643036, P60=0.010204594343850612, P61=0.04717813851681336, P62=0.21590273977458732, P63=0.33777166503723804, P64=0.16736356670557945, P65=0.13610065235189658, P66=0.09103648388812087, P67=0.017475176837528427, P68=0.0029145506478663796, P69=0.001802052659256526, P70=0.002505801244936321, P71=0.004713880253061674, P72=0.012839339977420141, P73=0.03251963101213347, P74=0.10192636659319466, P75=0.18938713750012726, P76=0.1482521631438608, P77=0.2633628380948213, P78=0.5177438881709673, P79=0.8647327238432029, P80=0.8434927082888964, P81=0.4640682314761912, P82=0.1896052207972509, P83=0.09733795835994528, P84=0.19001890300110752, P85=0.2089787307380799, P86=0.15622275109650202, P87=0.13448747084317667, P88=0.05670840747465251, P89=0.08394602367431925, P90=0.25364885659408937, P91=0.25291196438166086, P92=0.16077252595438216, P93=0.1941172516177718, P94=0.20446786669407074, P95=0.0387272896033382, P96=0.006608992707878157, P97=0.00824160076950515, P98=0.008484432084971691, P99=0.008391864630684386, P100=0.005363010749934356, P101=0.010171353328226126, P102=0.019018177193935948, P103=0.0197187601023425, P104=0.04089632561363416, P105=0.02990920394805613, P106=0.041034019255570466, P107=0.029742381200667127, P108=0.025944438900001334, P109=0.03527630719806006, P110=0.0229129695946249, P111=0.00547401892176091, P112=0.005137182362504512, P113=0.003054427032374707, P114=0.0014631511930882745, P115=0.0003792515347370796, P116=0.0016793238207814432, P117=0.003931298994065111, P118=0.0030971107781202064, P119=0.0011540936626913982, P120=0.0016610912186167427, P121=0.0010881804372128447, P122=0.001705034699106443, P123=0.0010889686727073775, P124=0.0003836308046158704, P125=0.0010110303761979825, P126=0.0011849817093535421, P127=0.001568360177681048, P128=0.00473578185543707, P129=0.0067251644910162715, P130=0.005380087654154424, P131=0.004052241811424979, P132=0.0015417699522686532, P133=0.0010949262226093986, P134=0.0010153239554324916, P135=0.0011908948614455335, P136=0.0023315373629223747, P137=0.005804176790319739, P138=0.01590308321411043, P139=0.017733692662964845, P140=0.0060540255797262256, P141=0.005335206570301938, P142=0.0037176917488011005, P143=0.004699856708996125, P144=0.0015243680360042474, P145=0.001063088539556672, P146=0.00019152616105466743, P147=3.5957150129652486e-05, P148=1.2869927215078561e-05, P149=2.5737204106335485e-05, P150=9.361916874246775e-05, P151=0.00023089473596551744, P152=0.0003117847413400234, P153=0.0003026736721225365, P154=0.0006617538027549088, P155=0.00039300700909925706, P156=0.0004951242673680075, P157=0.0003957579149627712, P158=0.000286926917327911, P159=0.00013094687744962379, P160=6.563165615493926e-05, P161=4.434469708342497e-05, P162=6.445409751439064e-05, P163=8.768033816274837e-05, P164=0.0001025528003630943, P165=9.945519451062564e-05, P166=9.346378821074936e-05, P167=5.752304011740171e-05, P168=4.0548033274530055e-05, P169=3.70315422040832e-05, P170=3.941629053265202e-05, P171=2.5813643738117084e-05, P172=2.9695204183566216e-05, P173=4.5384995520789886e-05, P174=4.815067861814297e-05, P175=5.216869171489493e-05, P176=6.11968476907526e-05, P177=4.694876642598547e-05, P178=1.6277525244550792e-05, P179=1.4578170618951787e-06),Row( P0=-697.9955862271136, P1=76.36074750991675, P2=-5.494392844150096, P3=16.836322987300157, P4=-15.394541242842712, P5=-12.691128618640224, P6=-2.1477138716989606, P7=6.263217899468495, P8=2.4017699057134254, P9=-9.998841062500563, P10=0.6700580548091201, P11=-3.9275639903471133, P12=-6.999948815299864, P13=-3.8264013894687445, P14=-2.9177851672685016, P15=-1.4802024107087408, P16=0.07009428401128975, P17=-0.46632313655196095, P18=-0.6287237664798723, P19=-4.326834820413381, P20=-1.6984834747347233, P21=-3.9974824602906445, P22=-5.502865714204578, P23=0.07821857086910161, P24=-6.277454220938514, P25=-0.7149278693237533, P26=-3.192698085968004, P27=0.16752490820788002, P28=-2.7223198500402015, P29=-1.4142969987352811, P30=-1.7907584320764163, P31=1.2249878448540765, P32=-1.695711080398761, P33=3.57404966168638, P34=-0.8181817521172261, P35=0.2500851624932497, P36=-0.9272083704672993, P37=0.07784619915583692, P38=1.2955388268015033, P39=4.005957798855442, P40=0.757025308100214, P41=0.7476856740751558, P42=0.7738745481811639, P43=0.7370963433734076, P44=0.6427760399224723, P45=0.618952801712439, P46=0.6234820083004705, P47=0.6305728140995979, P48=0.7016448498037715, P49=0.7124219277151004, P50=0.7426607614434456, P51=0.7799867261994793, P52=0.000277676886555014, P53=1.0252702135544418e-05, P54=6.030734720631759e-06, P55=0.00011602270554666743, P56=0.0006214188431790645, P57=0.0029150368233611086, P58=0.002168620080704644, P59=0.0016677361096048204, P60=0.000519572390120345, P61=0.00029668584348678995, P62=0.000481880852380566, P63=0.0013290736903380372, P64=0.0015168846236750621, P65=0.0012378153173344599, P66=0.00403459412590498, P67=0.00807562389405702, P68=0.002671071563701986, P69=0.0009231351778241092, P70=0.002251922850519403, P71=0.0016015324794676838, P72=0.000468429640546012, P73=0.0006825207508821978, P74=0.0016640960653003557, P75=0.00237870458135274, P76=0.0016489635342091511, P77=0.0029866599286821056, P78=0.002140459765284861, P79=0.0014934731629022478, P80=0.0020465107521945845, P81=0.002750295077472159, P82=0.0051260052883493985, P83=0.005364370883946756, P84=0.0019536303309785463, P85=0.0011626344963333647, P86=0.0007605356502160667, P87=0.001078414105275972, P88=0.0025457158468009857, P89=0.012845453209765602, P90=0.01163782008971605, P91=0.001124043471602778, P92=0.0003405860154829214, P93=0.0009061447597226728, P94=0.0017442066031604735, P95=0.003653684969968696, P96=0.0009220907285420237, P97=9.174159263875694e-05, P98=0.00015382736336114954, P99=0.00044897713409930185, P100=0.0005998854525095685, P101=8.455931748847547e-05, P102=6.956702975390375e-05, P103=0.00014229853852641767, P104=0.0004782525399077282, P105=0.0001514978904941263, P106=0.0001650710254270675, P107=0.0004886045191668796, P108=0.0008156023815067111, P109=0.00010941007700048861, P110=0.0002846328442080855, P111=0.0027484248396784467, P112=0.0006584057685614241, P113=3.831662321402001e-05, P114=0.00015624527498100222, P115=0.00021904293261314887, P116=0.0001178356003722036, P117=0.00019507367709105698, P118=7.144442572129153e-05, P119=5.704942071362849e-05, P120=6.260453569895149e-05, P121=6.776942776930687e-05, P122=5.02950766252293e-05, P123=5.5680887670835304e-05, P124=0.00011181266044132386, P125=0.0002897655809502365, P126=0.00025767745989996516, P127=0.0003197962609459273, P128=0.0002278520771646548, P129=7.667646112864155e-05, P130=3.630179302830404e-05, P131=1.6770746859758622e-05, P132=1.571684944677595e-05, P133=1.1982085729394375e-05, P134=1.0857116068601246e-05, P135=1.4076642640810521e-05, P136=1.3118201242231176e-05, P137=2.2671103608191024e-05, P138=2.7848832195648013e-05, P139=2.6511603347941914e-05, P140=3.567779042471135e-05, P141=1.9929477103230145e-05, P142=1.7345339010489302e-05, P143=2.0142383029052003e-05, P144=2.294490627312207e-05, P145=2.319235791893279e-05, P146=3.686838279098679e-05, P147=1.7288630814621002e-05, P148=2.47693783056897e-05, P149=3.692956797732099e-05, P150=9.081960461349025e-05, P151=0.00020675902738021194, P152=0.000312957563936909, P153=0.0001062057223446712, P154=3.479013597503887e-05, P155=2.5368546417733023e-05, P156=2.4658856088678087e-05, P157=1.798675476925594e-05, P158=1.4370483219703702e-05, P159=1.556237958839167e-05, P160=1.209620590724848e-05, P161=6.675886473737486e-06, P162=5.822029693847994e-06, P163=1.1018549780799174e-05, P164=6.824826157132444e-06, P165=5.2027216692375715e-06, P166=4.476560968470005e-06, P167=2.583705177610954e-06, P168=1.6820059790636468e-06, P169=1.045709440940399e-06, P170=5.956696066840545e-07, P171=6.303234221011278e-07, P172=1.1139251409712617e-06, P173=1.4678041373647485e-06, P174=3.4594144860606324e-06, P175=5.064812158575918e-06, P176=5.3117475631855995e-06, P177=4.678338481253496e-06, P178=1.801679159381169e-06, P179=1.5041093564726932e-07),Row( P0=-775.0547748466283, P1=80.09949526615753, P2=9.37939408723468, P3=12.672829270779502, P4=-0.8352193365175632, P5=-6.739013284466567, P6=-0.6162729751078805, P7=4.9347912436548285, P8=-1.590725394380231, P9=-4.575216501856474, P10=0.043916345813780074, P11=-0.9696935014363977, P12=-5.95493240405268, P13=0.5513770493990945, P14=-5.323063195514273, P15=-2.3272139765489506, P16=-1.7516068365736692, P17=-1.8703883988919345, P18=-2.0583535246050455, P19=-2.9959648720684164, P20=-1.5297312048963536, P21=-1.7308614161069924, P22=-6.630148908782008, P23=1.5426391819909766, P24=-5.555700797260197, P25=-5.032844778964913, P26=-4.41281039490634, P27=-1.8422861607451426, P28=-6.863794148373963, P29=-2.1599962030894946, P30=-5.333250583529341, P31=-1.726374111602061, P32=-2.339774002337655, P33=-0.7482048176641507, P34=-1.1070366997940924, P35=-0.008138261561906803, P36=-0.45356963617209317, P37=2.644698777698348, P38=0.7652924443828053, P39=2.0625134153527847, P40=0.7294557661029205, P41=0.7398888096549682, P42=0.7160744165305498, P43=0.7093221693126858, P44=0.7210992505223817, P45=0.7280604982368152, P46=0.7470265440304688, P47=0.7212115870160195, P48=0.7246022656747083, P49=0.7528717903506847, P50=0.7174045354784399, P51=0.6992472157490786, P52=5.204007570114081e-06, P53=3.860243924571453e-06, P54=1.2862029540315531e-05, P55=0.0003040599026444227, P56=0.0021025688150144804, P57=0.006168881421903843, P58=0.004640472168512931, P59=0.0033004053034933587, P60=0.0006525839810724333, P61=0.00016997705688325944, P62=0.0007792512760854357, P63=0.001764012318729559, P64=0.0019559265066214134, P65=0.003973254547400747, P66=0.005845790549258417, P67=0.005333628158637285, P68=0.0014130415441255667, P69=0.0003402216640503502, P70=0.000255100793407985, P71=0.0002600107235622124, P72=0.0006420620109293389, P73=0.0014308152272214424, P74=0.003888691755362011, P75=0.005377040043789888, P76=0.0016110929733549803, P77=0.0003241972905985796, P78=0.0004589670454721616, P79=0.0006650626749217753, P80=0.00045259694703005007, P81=0.0004663453593234011, P82=0.0012377022512076723, P83=0.0010517417948384732, P84=0.00026763650363484175, P85=0.0002271950159470426, P86=0.00020039970401851254, P87=0.0002841263000397304, P88=0.00019538161736113338, P89=0.00011943603492396393, P90=0.00018357680560296036, P91=0.0002904161168218152, P92=0.0001773379870563809, P93=4.3423789852780505e-05, P94=1.834813868472925e-05, P95=4.1682002953947144e-05, P96=5.847515495071816e-05, P97=3.627192771449316e-05, P98=1.7696870674117993e-05, P99=4.218131868301013e-05, P100=7.017284999100071e-05, P101=1.4966015411083363e-05, P102=4.350848770482935e-06, P103=2.3924973819585114e-05, P104=1.5115612243806236e-05, P105=4.123099595068699e-06, P106=2.630897748435574e-05, P107=5.325788049112627e-05, P108=1.0169701814009262e-05, P109=9.937228612371006e-06, P110=1.3386956885037485e-05, P111=1.1424366532285747e-05, P112=1.1430289907697902e-05, P113=6.6241861674601044e-06, P114=2.9168210108385473e-06, P115=3.519270690652025e-06, P116=6.903054471052516e-06, P117=9.999153023064039e-06, P118=8.609415503554503e-06, P119=2.0017578744631088e-05, P120=1.5918777531406887e-05, P121=4.426762767708487e-05, P122=1.3000526231850822e-05, P123=1.2061384042993643e-05, P124=8.4970913762435e-06, P125=2.6885130389281106e-05, P126=1.2760662594259324e-05, P127=2.0375815147054457e-05, P128=5.067791431528137e-05, P129=9.308254224800385e-05, P130=2.1881020928430866e-05, P131=1.540391527808151e-05, P132=1.4125331755125522e-05, P133=9.033471837408003e-06, P134=4.6803869572981735e-06, P135=3.7485170103438215e-06, P136=3.6322562323334773e-06, P137=3.0157926593623596e-06, P138=8.045969066108698e-06, P139=6.191489021824751e-06, P140=6.317739230096012e-06, P141=3.778318361396807e-06, P142=2.614285931348835e-06, P143=1.9042002843807742e-06, P144=4.929387632277901e-07, P145=4.982965256955576e-07, P146=7.816630880638501e-07, P147=1.9207310886419982e-06, P148=3.5767768689802163e-06, P149=4.780131377194722e-06, P150=9.155551163315882e-06, P151=1.214118873222525e-05, P152=1.4323080051237908e-05, P153=1.419324028175958e-05, P154=8.916965102030848e-06, P155=9.421639007888836e-06, P156=7.320353814572795e-06, P157=2.1722210529572093e-06, P158=1.464931186530207e-06, P159=1.1516560151063311e-06, P160=1.057343748926241e-06, P161=1.2681503225167528e-06, P162=1.7969146088541723e-06, P163=1.3327752953043837e-06, P164=1.6368935717846782e-06, P165=1.3708051400788125e-06, P166=1.4219919483915023e-06, P167=8.468361276678127e-07, P168=5.221971870561301e-07, P169=9.354780634441637e-07, P170=8.740511212326418e-07, P171=1.0995424959419685e-06, P172=1.985740775085908e-06, P173=2.3257565265240825e-06, P174=2.7206303508712683e-06, P175=2.207964452633478e-06, P176=1.6753802426088543e-06, P177=1.7610959288208694e-06, P178=6.287379253709685e-07, P179=7.93408514614643e-08),Row( P0=-602.0875461524471, P1=75.53331247340918, P2=-6.261650079446079, P3=15.16652134811953, P4=-10.112880835092561, P5=-20.029541405005546, P6=1.187296949299213, P7=2.773190592410945, P8=-3.5526750540536445, P9=7.66908788373401, P10=3.277504396816698, P11=1.9859420592785049, P12=0.9127110622547312, P13=-3.41991213653402, P14=-1.0292117500847613, P15=4.741652401875771, P16=-1.2173736682120502, P17=1.0172799165938038, P18=2.34650949803983, P19=-1.531990915790635, P20=2.587044767167188, P21=-1.8116990287059958, P22=-1.2294533559084238, P23=9.487548280638086, P24=-0.6916505240543032, P25=8.520894165491537, P26=4.4746802654129105, P27=10.510449810796707, P28=3.167139352322498, P29=7.502939333705373, P30=4.809170879634803, P31=7.12798934463368, P32=4.281297473565685, P33=5.364298031447818, P34=3.91991322033201, P35=0.695865699358761, P36=1.174079252530919, P37=2.2526148531070653, P38=-0.04890893290349488, P39=2.979203637888054, P40=0.7009451897827083, P41=0.6967883432386592, P42=0.7196014682132152, P43=0.691320271488971, P44=0.7102301712443302, P45=0.6954476589799671, P46=0.6502381156004268, P47=0.6415683822435131, P48=0.6533114050234786, P49=0.7137121719561229, P50=0.6905611094708083, P51=0.6615920440389029, P52=0.001481242389051473, P53=0.00022233193696147302, P54=4.752353780142136e-05, P55=1.9224724572008723e-05, P56=4.875830955262671e-05, P57=0.0003282648516458608, P58=0.001173017365679006, P59=0.007979590880280543, P60=0.023797093464251847, P61=0.032236263039733336, P62=0.03633939826953413, P63=0.014710177596636462, P64=0.0017700577153376693, P65=0.0003813065300790051, P66=0.0021286344010872926, P67=0.005744487648824958, P68=0.008234462961443412, P69=0.01209910041541628, P70=0.017656243322543834, P71=0.016045951960226523, P72=0.012894159315704935, P73=0.008894160878546676, P74=0.00559896523800991, P75=0.005816124700959829, P76=0.006401297336407193, P77=0.01471603559635434, P78=0.015397252513054308, P79=0.02306678751073925, P80=0.05067525450489252, P81=0.07968093960288093, P82=0.14049947695155665, P83=0.08263745613831991, P84=0.04457515636013155, P85=0.0282209691633602, P86=0.01622616861696196, P87=0.008147832422948368, P88=0.011998777201632865, P89=0.013428194290116597, P90=0.01738060757057655, P91=0.017058241369655554, P92=0.019822838100536207, P93=0.007246730212483789, P94=0.006139296943533473, P95=0.00493056044006051, P96=0.001840386372297572, P97=0.001971084293174184, P98=0.002817849776135677, P99=0.0028653971065436337, P100=0.0016037262169627414, P101=0.002050474818370125, P102=0.0018746706173449173, P103=0.0015037746196396367, P104=0.002134274200136525, P105=0.003282025301111323, P106=0.004101302513624866, P107=0.0041202374264717855, P108=0.0024750716574688074, P109=0.0012746427949859587, P110=0.0006564431090163069, P111=0.0011624562782283543, P112=0.0006239376591672363, P113=0.0001938184475528322, P114=0.00011880863760507909, P115=7.131757695176213e-05, P116=0.00014494624774967523, P117=0.0002108716381919123, P118=7.289547781988977e-05, P119=7.60838191250845e-05, P120=0.00016544445703981978, P121=0.00013784310480124333, P122=0.00017568399164107933, P123=0.00016242500783480445, P124=0.0002038926130253298, P125=0.00028832775121973383, P126=0.00023300452486691966, P127=0.0003268985718210626, P128=0.0004692692611364374, P129=0.0004099615077315079, P130=0.0004850108092761362, P131=0.00045551789012373287, P132=0.0003581951338506785, P133=0.00033922150553227377, P134=0.0002415485824200086, P135=0.00033055467446632295, P136=0.00033257383754284694, P137=0.0005157214841512743, P138=0.0005111520080495901, P139=0.00031990122513363026, P140=0.0003049618910749583, P141=0.00015945293358470393, P142=0.00012438882271526038, P143=6.795596055479706e-05, P144=2.9408717984582744e-05, P145=1.6651700920969266e-05, P146=1.2561887152873675e-05, P147=6.9432091405169124e-06, P148=1.3592297117026948e-05, P149=2.665407802792092e-05, P150=4.7759395500213574e-05, P151=4.032252236878052e-05, P152=2.9840312966941197e-05, P153=5.2636203894819616e-05, P154=9.985885957084968e-05, P155=9.825137252019372e-05, P156=5.6306295793684975e-05, P157=4.053471769430551e-05, P158=1.48520615705389e-05, P159=1.2189637159261222e-05, P160=6.775205526262453e-06, P161=9.622963481921106e-06, P162=1.1467701066738363e-05, P163=1.3522482957128674e-05, P164=1.6356568503225138e-05, P165=1.3380362439845874e-05, P166=1.0555619364189783e-05, P167=1.2225812296692098e-05, P168=6.740062395993253e-06, P169=4.918894590600457e-06, P170=3.0680721180479007e-06, P171=3.3449152625074524e-06, P172=5.939740273792092e-06, P173=1.5305907170764825e-05, P174=1.5125978897457005e-05, P175=1.4152592242512291e-05, P176=1.2341331045797047e-05, P177=7.35362769342964e-06, P178=2.5082179676076856e-06, P179=2.556697945157052e-07),Row( P0=-718.4671844992671, P1=72.69063410348147, P2=-0.060719194519611455, P3=10.359091449643447, P4=-2.2326287636123126, P5=-12.859707955207277, P6=-3.6561727853120356, P7=3.489429041631892, P8=-0.16017816879249577, P9=-2.6412516880683814, P10=-3.477757708143721, P11=1.319179814501673, P12=-7.962434982787159, P13=-1.2970696325745332, P14=-3.5080266878252875, P15=-2.243342856314643, P16=-0.9567149404899064, P17=-2.080276019881906, P18=-1.929921222864337, P19=-1.7921616737462027, P20=-3.0657955744060343, P21=-2.010721447986284, P22=-5.082247773488054, P23=0.5217157147777367, P24=-3.5535366926635006, P25=-2.635439778780405, P26=-4.034198403067695, P27=-0.45424337942943843, P28=-5.187060556018005, P29=-1.878798400545451, P30=-2.023579365436663, P31=-0.7712390882998638, P32=-0.14503887382165057, P33=3.3483757277003523, P34=3.4997084921409187, P35=5.22603031617264, P36=4.614241082243942, P37=5.330766323243346, P38=2.449367872849842, P39=2.744033662242886, P40=0.6725472179548608, P41=0.6560759070764403, P42=0.6528269998975141, P43=0.6199239490502468, P44=0.6210562902536546, P45=0.6360898508688902, P46=0.6123766222530225, P47=0.6150080123775477, P48=0.6756406492353919, P49=0.6702376057036072, P50=0.7103904297302641, P51=0.7018601140581036, P52=1.8815170576397237e-05, P53=6.938748464050438e-06, P54=1.159822574508349e-05, P55=0.000157006298327484, P56=0.0007613722407607457, P57=0.004208862058860757, P58=0.008441125891166351, P59=0.007445302478543098, P60=0.0017048144313076671, P61=0.000336647091413478, P62=0.0004343672949429997, P63=0.0014268960626097722, P64=0.004570806137622775, P65=0.012658302651463936, P66=0.018847496424116518, P67=0.008983045426014448, P68=0.002592146549742459, P69=0.0013907604323238857, P70=0.0011902917226176636, P71=0.001571957298427918, P72=0.003173812328903235, P73=0.004411110995331496, P74=0.006404974495880287, P75=0.003803953830889876, P76=0.0022725337432153722, P77=0.004947660359699159, P78=0.004965056140959695, P79=0.004328875133333249, P80=0.0038287613185806805, P81=0.007356777617566974, P82=0.015121746210880605, P83=0.01072724860779171, P84=0.008338461067946569, P85=0.01099998416887495, P86=0.0069086007420181535, P87=0.004548087447062745, P88=0.013797305013592906, P89=0.00809178969772499, P90=0.0031514505275159316, P91=0.0017415302010111511, P92=0.0006257772290040934, P93=0.001699851396883393, P94=0.002281507669499137, P95=0.0008119502238339827, P96=0.0006640586949291492, P97=0.0001820171011933327, P98=0.0004167470574248401, P99=0.0007484973492827135, P100=0.0005343782809755692, P101=0.00013495818423410875, P102=0.00014909977255946698, P103=0.000554935853266346, P104=0.0004476565402756245, P105=0.0001251078111568569, P106=0.00041248776085081314, P107=0.0006216996971054763, P108=0.0002817473203825328, P109=0.0004856265945156589, P110=0.0006167526377211296, P111=0.00024023721401872822, P112=0.00015277186876128503, P113=0.00013371038052391296, P114=0.00015556153939538618, P115=0.0003368067971807959, P116=0.0001259430289790889, P117=0.0001642658429490581, P118=5.689161603344259e-05, P119=0.00019949415446435636, P120=0.00015839875173434098, P121=7.479641755958678e-05, P122=9.854562482204717e-05, P123=0.00016572768824599923, P124=0.0002920038967017287, P125=0.00020627950859281193, P126=0.00013595624934784517, P127=0.00022332094369748273, P128=0.00019828523712629855, P129=0.00016593973619187416, P130=0.00011415049838516023, P131=5.410245464548527e-05, P132=2.0227400488633404e-05, P133=1.4911334407689585e-05, P134=1.3093140526264348e-05, P135=1.0440571188569328e-05, P136=1.3390459522144663e-05, P137=1.9609728558446636e-05, P138=2.7871128928947452e-05, P139=7.3861058695237e-05, P140=5.085606590090372e-05, P141=3.0405880548087856e-05, P142=1.5784270663776117e-05, P143=8.600336911776922e-06, P144=7.051125642437189e-06, P145=1.1365821967426436e-05, P146=1.9736481734625713e-05, P147=3.29554585853772e-05, P148=3.3447724503508124e-05, P149=9.145392694495937e-05, P150=0.00012332422937906578, P151=0.0002059901866377127, P152=0.00033258472549629317, P153=0.00029593822385296837, P154=0.0001100337394332721, P155=3.406988741126288e-05, P156=3.204574341297547e-05, P157=2.230996482653274e-05, P158=2.2660651982913638e-05, P159=1.2395566169028202e-05, P160=9.349407394869038e-06, P161=1.4091945138230857e-05, P162=2.15934064382425e-05, P163=2.4583785544121835e-05, P164=1.5297830963787105e-05, P165=1.2363659883847443e-05, P166=1.3058710246450116e-05, P167=1.6272185516742616e-05, P168=9.46431972140821e-06, P169=6.69246935964033e-06, P170=1.0544843921626262e-05, P171=1.0108981270452191e-05, P172=9.846629730095367e-06, P173=8.319237786358145e-06, P174=7.715573807962691e-06, P175=7.88218984753324e-06, P176=5.0146917239363816e-06, P177=2.9782539696851143e-06, P178=9.513739419237363e-07, P179=5.9503761667370676e-08),Row( P0=-732.2111913315387, P1=67.20510895928356, P2=1.5911716406515217, P3=8.716750053805455, P4=-5.286582334974365, P5=-2.913911817008277, P6=3.433407267123496, P7=3.2341298698779695, P8=3.006433396902333, P9=0.5631651884211979, P10=-0.20095256167965658, P11=3.3334979639560687, P12=1.0249018198189472, P13=-0.5056878999569795, P14=-1.4063502677940134, P15=3.9621296187614425, P16=4.443960692055803, P17=1.4270906357295094, P18=0.037070503546528855, P19=3.00969364086502, P20=-2.5901392810441215, P21=1.3189959726075573, P22=-0.06450602491337352, P23=2.3672329250535693, P24=-2.0042132832579007, P25=2.791243835695291, P26=2.130927717071082, P27=4.1465562767472415, P28=1.898504486994976, P29=4.108100095288595, P30=0.5597270588643866, P31=4.839426293598674, P32=0.3149649207848345, P33=3.476528109275375, P34=1.6402019862593662, P35=1.6997312572176262, P36=-0.054820029559290384, P37=2.7016956165946797, P38=1.6848987434860572, P39=0.4440026235292647, P40=0.7205497856132616, P41=0.7056626467411874, P42=0.7167855609872475, P43=0.7349420293907754, P44=0.706697968842793, P45=0.7608501474006274, P46=0.673010666165376, P47=0.6758552672688062, P48=0.767227237905568, P49=0.7346654085642148, P50=0.720969037303046, P51=0.7111469675285678, P52=9.974220010926779e-05, P53=1.4759611529444993e-05, P54=1.2914380604665452e-05, P55=4.007596451944502e-05, P56=0.00012306769777528949, P57=0.0006475180987770389, P58=0.001668826219539202, P59=0.0034220880519522747, P60=0.006455386035322172, P61=0.0061701105188936515, P62=0.0024475647741889622, P63=0.0008665381914739797, P64=0.0007665139291987634, P65=0.004340932452889051, P66=0.009286680992154378, P67=0.010254967290620262, P68=0.008917547174402076, P69=0.01016845698148819, P70=0.01603203030174641, P71=0.008759686533636056, P72=0.003059571751862806, P73=0.003133698038367159, P74=0.0032999043154306744, P75=0.00366326961160607, P76=0.005128861716224758, P77=0.008203454208745317, P78=0.00918486908700786, P79=0.012760630571977915, P80=0.009528896749793458, P81=0.01004271578640242, P82=0.012492772419915388, P83=0.009420978190809464, P84=0.00838113807178065, P85=0.008669632347756754, P86=0.004285264845294916, P87=0.003950246971917567, P88=0.011022269641935603, P89=0.010516809372998601, P90=0.007859136188153402, P91=0.0026191919809363897, P92=0.0019118172675487975, P93=0.0019475084253526953, P94=0.006115107772869018, P95=0.009581917176618424, P96=0.0031583469686562293, P97=0.0025203701995044387, P98=0.0016775647872420106, P99=0.0008485046978648749, P100=0.0028796567565076846, P101=0.002292862291264255, P102=0.0009845015567772264, P103=0.00029819980553490345, P104=0.00030771168072445733, P105=0.0007411020432003318, P106=0.0008403917018901172, P107=0.0008139652029991091, P108=0.0006070493350884929, P109=0.0006032676352849173, P110=7.675490135542795e-05, P111=0.00028450215183804795, P112=0.0002196340450659275, P113=0.00010547271383522983, P114=4.9693886309366984e-05, P115=3.250174894499749e-05, P116=0.00014685184398415528, P117=0.00026063191294765486, P118=0.00011191859478636212, P119=0.00022439357647597173, P120=8.955441121345267e-05, P121=0.00011352839453585174, P122=8.61943076510125e-05, P123=7.406214220012902e-05, P124=9.371285599951564e-05, P125=5.6304763632900086e-05, P126=5.462086773348758e-05, P127=4.655814598522552e-05, P128=4.9650259192585e-05, P129=4.789152711213494e-05, P130=4.251288265552528e-05, P131=0.0001050692549070744, P132=8.762602616884749e-05, P133=0.00011183332621019422, P134=0.00015431709485816674, P135=9.380296798497928e-05, P136=5.926152262457701e-05, P137=5.324133798522882e-05, P138=3.5836318745195345e-05, P139=4.102471379968943e-05, P140=3.521739338091326e-05, P141=2.6547372165555953e-05, P142=4.106505715930873e-05, P143=3.376851323504522e-05, P144=3.137929461592853e-05, P145=2.0622267349801687e-05, P146=2.6181525958346533e-05, P147=2.675069454061676e-05, P148=4.8938928922236696e-05, P149=9.371283361309147e-05, P150=0.00014580124073414966, P151=0.00019594178442725887, P152=0.00018066659247487746, P153=0.00017308040852703652, P154=7.518658645439874e-05, P155=2.2940022885979662e-05, P156=2.9240534562376797e-05, P157=1.8943194317183907e-05, P158=2.444120137710908e-05, P159=2.524511824652411e-05, P160=3.4327723479495536e-05, P161=2.62356193916425e-05, P162=2.47291587440988e-05, P163=1.61521380727406e-05, P164=6.819088995778395e-06, P165=7.996377370549085e-06, P166=9.96795424103335e-06, P167=9.085358340131272e-06, P168=9.687525433178777e-06, P169=7.095105620455073e-06, P170=5.325378536779909e-06, P171=5.396222952086927e-06, P172=6.026015590659155e-06, P173=1.2839981706797795e-05, P174=1.875995082470556e-05, P175=1.6576049473441892e-05, P176=1.0248547071356084e-05, P177=6.121067603164815e-06, P178=3.6905432348118108e-06, P179=3.2288210108189796e-07),Row( P0=-678.6934400548917, P1=54.752824216323496, P2=-6.042645676367279, P3=1.3597185891171018, P4=-9.48661011127742, P5=-5.8516519645068055, P6=1.7699361346168467, P7=-0.14653076828318198, P8=-2.751369366839565, P9=-0.25704547416832235, P10=0.9248830543871156, P11=0.3223791863278425, P12=-1.168427016579046, P13=-1.1687017986697763, P14=-3.0118044266709862, P15=3.287744573387509, P16=3.1117257244072096, P17=4.895160496006523, P18=5.332450058718043, P19=3.6923584544200905, P20=2.706003185112576, P21=5.872251059706397, P22=1.8900169578189163, P23=5.2223294854010165, P24=-0.5988665404578866, P25=5.897866399366535, P26=5.226633679424715, P27=5.380263425572353, P28=3.4406943427364673, P29=5.82932283686734, P30=1.8128525185586186, P31=4.355293750019498, P32=0.7917109863396701, P33=1.9644834743072237, P34=1.1363650308558582, P35=2.5086264917279872, P36=0.5774778249555285, P37=1.9086144714247888, P38=0.6712241790350462, P39=0.6739062963048014, P40=0.6721878836410191, P41=0.6439174057051931, P42=0.6972506753837913, P43=0.7362416282294344, P44=0.7046942862928658, P45=0.7545063440375486, P46=0.6932091077604129, P47=0.6404144005406188, P48=0.6822666934026725, P49=0.7611125263139642, P50=0.7455031561361285, P51=0.7028708805361262, P52=0.0001246130122779275, P53=3.0631659517086416e-05, P54=7.511689126988271e-06, P55=7.0632210517990475e-06, P56=7.251297532982307e-06, P57=2.0623240540261624e-05, P58=0.0001380123621362817, P59=0.0009430851607900646, P60=0.003981032341720378, P61=0.005592719878522073, P62=0.010626298069976796, P63=0.015171954735289827, P64=0.009132473041496758, P65=0.011083903862482305, P66=0.018369317926008266, P67=0.023780906415180367, P68=0.010086367381029863, P69=0.00741974654488818, P70=0.014161722639266492, P71=0.008825699467571682, P72=0.006945609643506632, P73=0.01123139370123355, P74=0.01290296545217335, P75=0.011330604955079659, P76=0.011595014408862386, P77=0.01923357922701677, P78=0.04684029161959824, P79=0.08159670640932135, P80=0.04113289350309958, P81=0.02070572169895621, P82=0.02767513725550456, P83=0.02325427302392741, P84=0.01557888506339747, P85=0.00869598348877295, P86=0.003890016608251719, P87=0.003942480910725659, P88=0.006657784581301428, P89=0.0033725808675821745, P90=0.0020083568249937584, P91=0.0035970186784312626, P92=0.011516685277691022, P93=0.014201093816707266, P94=0.020498968224930143, P95=0.014876493743276679, P96=0.002808591777697443, P97=0.0017086157462818264, P98=0.0016203274132279458, P99=0.003347491035367084, P100=0.006198622151650739, P101=0.009615668509524087, P102=0.006469202415840723, P103=0.00839563926727773, P104=0.005146941999623673, P105=0.0007594211630609906, P106=0.00044941823420738405, P107=0.002711283855860616, P108=0.007503485193683303, P109=0.0054687104693548535, P110=0.0013021077094651954, P111=0.00032953728832831714, P112=0.00017809330907209698, P113=0.000159048656498261, P114=7.928145408189173e-05, P115=3.2240332744266334e-05, P116=4.845594551120187e-05, P117=0.00015352835397676283, P118=0.0001322781459823392, P119=0.00015945894188009108, P120=4.7268673175379815e-05, P121=5.881357334566398e-05, P122=0.00023359919050570637, P123=0.0002793518465620108, P124=0.00041474524665133103, P125=0.0005423899042451153, P126=0.00041021876482093207, P127=0.000296272361478304, P128=0.00022993106995820707, P129=0.00023118201447271778, P130=0.00016280002676282725, P131=0.0003722752364530525, P132=0.00021061028999479607, P133=0.0001542515136869492, P134=0.0001571490452218101, P135=0.0001219730021851207, P136=7.064099222069436e-05, P137=9.877479333641109e-05, P138=7.489576179329496e-05, P139=7.700197575235175e-05, P140=0.00010418904955380534, P141=7.408384680900977e-05, P142=9.06571613151548e-05, P143=7.773898716232335e-05, P144=4.1620369032209384e-05, P145=1.7592992902118472e-05, P146=8.70314731981919e-06, P147=2.3287473157792883e-05, P148=3.710931164191494e-05, P149=5.4730001108461785e-05, P150=8.548981478273471e-05, P151=0.00010560280111695131, P152=0.00016287333259247462, P153=0.00027368512549594563, P154=0.00013432294973623904, P155=0.0001070298881077108, P156=8.845781735361774e-05, P157=2.7046840063364393e-05, P158=1.6790832994528288e-05, P159=1.444639498356882e-05, P160=2.1654568803480292e-05, P161=2.8814736119201933e-05, P162=4.367630732128071e-05, P163=2.7526172699808163e-05, P164=1.9219618106287243e-05, P165=1.0727646549720559e-05, P166=1.0032422416015137e-05, P167=1.2786690549757127e-05, P168=1.0921004247959805e-05, P169=1.798852120971475e-05, P170=3.908889900444149e-05, P171=1.494498140033689e-05, P172=1.9492456060655653e-05, P173=1.9910889460115052e-05, P174=2.5911133123433592e-05, P175=3.4880286369465025e-05, P176=4.2992138379550404e-05, P177=1.545167033787174e-05, P178=1.5092034162769946e-05, P179=1.1233432124255057e-06),Row( P0=-683.0696248462177, P1=62.33143787765614, P2=-3.03385649331132, P3=2.2090742554737925, P4=-14.539259535395875, P5=-1.351386086956852, P6=4.271850313660387, P7=0.533882820516534, P8=2.140013430529228, P9=-1.1016907755448562, P10=-0.07700295323429102, P11=-3.6908645448874866, P12=-0.29441707798063055, P13=-0.013142322490733278, P14=-3.0292613394645262, P15=4.840718448469592, P16=0.7554186273805656, P17=-0.9629132848756978, P18=6.069717558771584, P19=4.996626547795236, P20=3.318074031738218, P21=4.959961369570792, P22=3.572044214924338, P23=4.740514617262369, P24=0.3444679782359854, P25=2.115900751720314, P26=1.1806857820514733, P27=1.76165162435699, P28=1.6910413447832622, P29=0.06603800186698619, P30=-0.8835968823477329, P31=2.1304119374171724, P32=-2.997175974318575, P33=3.007467571499639, P34=-1.387898348948698, P35=0.7542270439429992, P36=-0.593875953318371, P37=2.226289903035414, P38=0.8793239183699368, P39=4.0906209290141575, P40=0.7074286300910995, P41=0.7013224846567445, P42=0.745299749605544, P43=0.7637657381709316, P44=0.7525921628883384, P45=0.7668391456381121, P46=0.7372968666827032, P47=0.7471929400583261, P48=0.76797020841734, P49=0.7041705042631802, P50=0.6812728864213997, P51=0.6701215830396058, P52=0.00013932965541323873, P53=1.8463198410031947e-05, P54=9.234556104263006e-06, P55=2.63678500516233e-05, P56=0.0002353668817904342, P57=0.0011792357916116341, P58=0.0009620354115706777, P59=0.0006385568260916757, P60=0.0007601127173295693, P61=0.0023065707338347694, P62=0.008538806953363134, P63=0.011154425025230312, P64=0.007999067494107386, P65=0.003921815761543999, P66=0.001272234081951825, P67=0.0008585600130008104, P68=0.0006770541772241901, P69=0.000819253674369415, P70=0.0013823086183245624, P71=0.0011484038467912013, P72=0.0014537138709606657, P73=0.003032946672654103, P74=0.004229467034059658, P75=0.008055694533182906, P76=0.02251422262777317, P77=0.024094543656715782, P78=0.008393545739685098, P79=0.0062975174834331805, P80=0.0029356466632194694, P81=0.002104626917021147, P82=0.0035765675723200985, P83=0.004034148653667978, P84=0.005741519526536631, P85=0.0062461377123461635, P86=0.0038866295307701426, P87=0.0029395271366841865, P88=0.007008486196775764, P89=0.005048815264284863, P90=0.002588232786985611, P91=0.0038265017123931366, P92=0.002170016509805269, P93=0.0013499579733042713, P94=0.0009958661701214681, P95=0.0011811966826292106, P96=0.0008408417739873805, P97=0.0012203268140827733, P98=0.0014661762876710103, P99=0.0015897960872289102, P100=0.0016249830078789117, P101=0.0008752396414929119, P102=0.0002977328379553895, P103=0.0002706867263017863, P104=0.0007972626080397698, P105=0.0038690528590684327, P106=0.002572045646712866, P107=0.0015002915983435606, P108=0.00038431187875722543, P109=0.0001561846245013111, P110=0.00012537816825149763, P111=0.00017361705209985916, P112=4.470676200404225e-05, P113=6.273360876586245e-05, P114=4.3573245873073736e-05, P115=2.2453811809537243e-05, P116=2.2289118778707474e-05, P117=2.8627520616503347e-05, P118=5.242692097273823e-05, P119=6.170780277020888e-05, P120=3.543874244729172e-05, P121=5.7985390439173805e-05, P122=0.000245523002195884, P123=7.830736592802248e-05, P124=3.808042971435703e-05, P125=0.00030408291476737, P126=0.00018276437375496873, P127=0.00022089077675997034, P128=0.0003521473848303581, P129=8.110828783678183e-05, P130=7.944325851029557e-05, P131=0.00015264484294540479, P132=8.440306554651896e-05, P133=0.00018936343267525214, P134=0.00023250455006625212, P135=0.00013495317808154088, P136=0.00026038872708168106, P137=9.081002844112823e-05, P138=8.374123180696885e-05, P139=3.247016951455106e-05, P140=3.370135457786474e-05, P141=1.6794601704781112e-05, P142=2.050257784410938e-05, P143=2.005391054050887e-05, P144=3.461661524651683e-05, P145=3.4571772733531166e-05, P146=1.3787311165673318e-05, P147=1.266638758862807e-05, P148=1.558880174073846e-05, P149=3.66164154529603e-05, P150=6.501477350799611e-05, P151=0.0001077464650355382, P152=0.000107516637047388, P153=0.00010020975773878211, P154=9.87144491812277e-05, P155=3.5252016714441104e-05, P156=2.6803539546192593e-05, P157=2.825701693425484e-05, P158=4.3072067695493054e-05, P159=5.2510809839950366e-05, P160=3.355290026569654e-05, P161=3.210188453810891e-05, P162=2.214924696928554e-05, P163=1.5827058083124657e-05, P164=1.685651483375685e-05, P165=1.5575435878675492e-05, P166=8.665667098621143e-06, P167=1.2997123491593571e-05, P168=1.5517304832370674e-05, P169=1.3041551957023702e-05, P170=1.1733240159618893e-05, P171=7.092544910389655e-06, P172=4.242451697647376e-06, P173=4.265347611536729e-06, P174=3.631559128508852e-06, P175=4.335912013263292e-06, P176=4.642761020492533e-06, P177=3.910616323018438e-06, P178=2.094472268948589e-06, P179=2.9865065237313763e-07),Row( P0=-637.1789595526612, P1=84.46015778092476, P2=-8.312653651617023, P3=-7.7378538647963895, P4=-17.81273841496333, P5=-1.0775757007236173, P6=2.0500704132043763, P7=-1.3852629071721614, P8=-1.3025254865356268, P9=-2.471721039179936, P10=-3.4492544829894567, P11=-5.220913942965025, P12=-4.338537219932847, P13=3.194316917841669, P14=-2.2818942238290982, P15=1.0553302031306715, P16=2.0945278459408083, P17=-1.0314723226709497, P18=0.2851598360567019, P19=3.964197194335829, P20=-0.6165765956332996, P21=-1.9463693277430023, P22=0.986283210594761, P23=0.7589411473758356, P24=-2.396754010432715, P25=0.7416732903015334, P26=-1.8057003610546238, P27=3.0814781966584066, P28=1.7377220280661183, P29=1.5176059917055738, P30=-0.5792496214971236, P31=5.194058548576255, P32=-1.668413028751951, P33=3.65558177207566, P34=0.23419199822019113, P35=2.31298642721905, P36=0.40139712264669836, P37=1.8846828309333041, P38=1.4602500569919332, P39=3.0833968231858693, P40=0.7295124699235533, P41=0.7210561705993271, P42=0.7181900137189747, P43=0.751449009332451, P44=0.717075456851746, P45=0.7134736213681645, P46=0.7155153435253652, P47=0.7099931315132052, P48=0.6805972394258469, P49=0.6585512428968316, P50=0.6734275211467174, P51=0.6959762907683346, P52=4.605291833390869e-05, P53=1.8833702451004517e-05, P54=7.528251884336508e-06, P55=1.901048490560633e-05, P56=0.00037421165397333246, P57=0.002148549386859268, P58=0.0017016305098400007, P59=0.0017424853449370587, P60=0.0019854534622774146, P61=0.0019832478240648975, P62=0.002624122652544295, P63=0.006155181555734557, P64=0.006244345394454146, P65=0.0029879767316501053, P66=0.00254998647457569, P67=0.003910635190604806, P68=0.003851045923770539, P69=0.0033046054095953285, P70=0.004212034883013201, P71=0.00368153262087566, P72=0.0013096678644295475, P73=0.0016636901258857961, P74=0.002039537567092279, P75=0.002695067373201024, P76=0.005117605677514463, P77=0.0074073515740967774, P78=0.0046760308533896865, P79=0.0032894182229257427, P80=0.003919408714743613, P81=0.005438401819028106, P82=0.005333975041092438, P83=0.004421007454610748, P84=0.005554855520406607, P85=0.0062216357519075125, P86=0.005045916813562741, P87=0.004322640505758302, P88=0.014712437703091084, P89=0.02445835245507976, P90=0.014569493194587729, P91=0.009764279941868738, P92=0.008644054011377645, P93=0.006975192506662596, P94=0.007473998897868273, P95=0.009822975147534739, P96=0.0074309876233909465, P97=0.007809959155423615, P98=0.012091681691052375, P99=0.007620377366310027, P100=0.0025176993319325257, P101=0.0021363882450946316, P102=0.002595867100186874, P103=0.0027350452160347866, P104=0.003596803384624749, P105=0.006795429659335825, P106=0.005682224456117029, P107=0.0035352497579989248, P108=0.0022122445745493945, P109=0.002018662714480215, P110=0.0015540385747436277, P111=0.00039654404738547497, P112=0.0002337857543723336, P113=0.0007167860715594442, P114=0.00019510653835908402, P115=7.623609790353633e-05, P116=9.015123439840562e-05, P117=8.826191808312847e-05, P118=0.00015964882017971744, P119=0.00015369093442495313, P120=4.6812973811544645e-05, P121=3.258298816669263e-05, P122=4.672447059304574e-05, P123=5.889868360955847e-05, P124=6.476722303978792e-05, P125=6.943971405102736e-05, P126=4.824457160128659e-05, P127=7.630996054044646e-05, P128=9.22389972002566e-05, P129=0.00012438566247970268, P130=9.616425192158807e-05, P131=9.392793019395392e-05, P132=0.00011055061722954345, P133=7.906962232963533e-05, P134=6.958246672399246e-05, P135=5.285864375669583e-05, P136=5.2319053796506866e-05, P137=4.0479465836331725e-05, P138=2.6006822734342786e-05, P139=9.98457671590613e-06, P140=1.7616672593228293e-05, P141=2.3509191018434683e-05, P142=2.8999038224852084e-05, P143=3.3626849378564485e-05, P144=4.404815109302291e-05, P145=5.113341521505449e-05, P146=3.322574887168329e-05, P147=2.5359977435794496e-05, P148=3.255062705792262e-05, P149=5.6380396598731326e-05, P150=6.62475972267165e-05, P151=0.00010211389773528302, P152=0.00016324998600087464, P153=0.00014792914634887297, P154=0.00010951951493274517, P155=0.00012524060551793938, P156=0.00021077750944864084, P157=0.00023878197716369951, P158=0.00024411640559687204, P159=0.0002312306292164787, P160=0.0002525973538956162, P161=0.0002116819755361314, P162=0.00018224734144959602, P163=0.00011908481433589624, P164=7.590032177870435e-05, P165=0.00011966868433138414, P166=0.00013391569366179719, P167=0.0001548353035959804, P168=0.0001369694325556763, P169=9.528310552561424e-05, P170=4.706612784774834e-05, P171=2.057507255418899e-05, P172=7.54418052296267e-06, P173=8.460551604524571e-06, P174=1.345452559007578e-05, P175=1.5089577374289657e-05, P176=1.4155846794158467e-05, P177=1.0715780580111726e-05, P178=4.130759082825307e-06, P179=5.302117810781843e-07),Row( P0=-500.0309060590954, P1=34.73398263019542, P2=-28.067218057177076, P3=-6.982499758455402, P4=-19.728451639593228, P5=-17.94629079738611, P6=-13.651040191058534, P7=-5.575133699633519, P8=-8.99480053119528, P9=-7.868943630094314, P10=-3.613537579291378, P11=1.0583309725822783, P12=16.955274726575098, P13=14.059129072347385, P14=13.987061468188658, P15=11.651824203528353, P16=-0.4097635748329006, P17=1.0658248140020334, P18=-0.19351504308453732, P19=5.837677601396449, P20=3.4753216279493717, P21=0.29713554053437186, P22=-1.3727590881566125, P23=0.9374241518128971, P24=0.27521686540637885, P25=6.412499376793504, P26=3.8317365575764355, P27=0.4841328178602384, P28=-0.9233614669250597, P29=1.1804238078088913, P30=0.990386651999522, P31=4.245569684807807, P32=1.9576165089186024, P33=0.7921575164388698, P34=1.842074093898593, P35=4.271453784691068, P36=2.2453710888104177, P37=2.4892226071668335, P38=0.19602053357977964, P39=0.9391768776004876, P40=0.5475211942085377, P41=0.6010390548796333, P42=0.6792647472270773, P43=0.7091914624093093, P44=0.6979876669463952, P45=0.6546333239183998, P46=0.5397863079847344, P47=0.4900115314716512, P48=0.4590008465114685, P49=0.5109583507255457, P50=0.5385618055534784, P51=0.5282636940072234, P52=0.0006930113302395289, P53=0.00011103528942772926, P54=4.055903493329451e-05, P55=6.032395045745395e-05, P56=9.054843312157338e-05, P57=0.00014051414335009474, P58=9.695220712658894e-05, P59=8.435934755019505e-05, P60=0.00012429069282450094, P61=0.00019325905418523704, P62=0.0002755041570494668, P63=0.0004073585593368268, P64=0.0004739483366176047, P65=0.009127414634639676, P66=0.04371513369181567, P67=0.0775380886559943, P68=0.07493665393169738, P69=0.16170791045793495, P70=0.3123718694957592, P71=0.3542007841255712, P72=0.40033355818733846, P73=0.54973176827556, P74=0.46003833736885313, P75=0.23456486085282352, P76=0.0834093612750937, P77=0.02136472495974369, P78=0.006695773160826655, P79=0.005057351090018906, P80=0.0031565257055822084, P81=0.0028385496767772407, P82=0.004890977760300634, P83=0.0068063398946452735, P84=0.011592795118009503, P85=0.02628608981529457, P86=0.05211453916726882, P87=0.2294197159416112, P88=0.8786338480093875, P89=1.1062447109821132, P90=1.4326388299575863, P91=1.803339592319226, P92=2.2018981788302714, P93=1.1501059667630547, P94=0.8596951945284658, P95=0.2690167139638755, P96=0.04462501538697659, P97=0.01749813941602974, P98=0.004131166321595395, P99=0.03216959168877996, P100=0.1344534256120146, P101=0.12795586444361168, P102=0.08644818176929558, P103=0.09599950161085868, P104=0.054158010615267475, P105=0.03147736559592868, P106=0.1438835800812303, P107=0.4314983828906598, P108=0.1241090824361043, P109=0.07970971100760416, P110=0.022740185285918507, P111=0.2036735750121405, P112=0.2906308945630206, P113=0.13241684043415883, P114=0.032480734821353295, P115=0.02632628655628112, P116=0.04499222991265255, P117=0.08801029860866834, P118=0.1935957172081234, P119=0.07873901574104754, P120=0.010488554204617303, P121=0.012211070750473695, P122=0.023742590913862018, P123=0.04462798276676994, P124=0.020321892652422173, P125=0.00629174857104488, P126=0.011695537574984882, P127=0.02190481491050253, P128=0.008945611049608497, P129=0.016186403663903046, P130=0.007030043744676522, P131=0.009636823131152954, P132=0.018941007998106347, P133=0.029081940450655982, P134=0.01696757822333913, P135=0.0071074260906845155, P136=0.004577940224022049, P137=0.004044527867147592, P138=0.0023135769906913717, P139=0.007202781547635327, P140=0.009965173834178057, P141=0.0032450004190625503, P142=0.003483568104609163, P143=0.002100901455496471, P144=0.0012997269051634387, P145=0.0011787839877217484, P146=0.00034779248176638523, P147=0.0008107637334593473, P148=0.0009846890651362982, P149=0.0007674121943502075, P150=0.0009155950696245556, P151=0.0008721538581658417, P152=0.0011224141945953042, P153=0.001705316427085444, P154=0.002675990957297819, P155=0.003113082602776735, P156=0.004379177987165756, P157=0.004291565503054458, P158=0.003992992203293526, P159=0.0025974556692003477, P160=0.0012084681808775801, P161=0.0010023812033187552, P162=0.0005495865965592247, P163=0.0008182819869221128, P164=0.0009188160323586602, P165=0.0007205471803326108, P166=0.0008868274792191965, P167=0.0007190809671015557, P168=0.0006781129290085172, P169=0.0005511754271034928, P170=0.0006738963351679632, P171=0.0006700557318217987, P172=0.0007624137238197033, P173=0.0006721851702167891, P174=0.0010565759462094091, P175=0.0007806725381586906, P176=0.0006590252203917818, P177=0.0005790189063292989, P178=0.0002629189887202416, P179=2.3073802327331435e-05),Row( P0=-640.8729644150274, P1=54.34152264269915, P2=-9.03399532299627, P3=0.38742983303373707, P4=-11.554207489259406, P5=-8.889979087898723, P6=-0.101536033670119, P7=-2.1908657691877327, P8=-1.5908986052794687, P9=-1.2702043765364979, P10=-2.814366373604487, P11=-1.906348921898608, P12=-1.2432565366576085, P13=-2.1874830352372263, P14=-2.5360190506322864, P15=0.8840836140223916, P16=0.5038352087229826, P17=1.0937022180274047, P18=-1.6837096568176964, P19=1.4644844996427038, P20=-1.2332149380821573, P21=4.3745847292238516, P22=4.02952463830293, P23=6.370317706952406, P24=3.027044747397445, P25=4.669926666014234, P26=4.287036303600563, P27=2.2345743166302383, P28=0.8884870326141419, P29=2.4527520341262337, P30=-0.38560134959594794, P31=3.7709108840943, P32=-0.8630772145279182, P33=1.3730698506718901, P34=0.07207978315122085, P35=2.5222307413621308, P36=0.9912560969718832, P37=2.207420742356662, P38=0.8467107348818972, P39=0.7792103946037578, P40=0.671140899343826, P41=0.6619909806430005, P42=0.6776189099049893, P43=0.698689519968064, P44=0.7461222235156796, P45=0.8125185984341567, P46=0.7143686847272591, P47=0.6688522622604155, P48=0.7120889384969811, P49=0.6819228056081105, P50=0.6602129918821812, P51=0.662692668190467, P52=9.247008115031204e-05, P53=2.404857349277109e-05, P54=1.3787515615981186e-05, P55=3.3865760390465025e-05, P56=0.0001076283223448313, P57=0.0006949523786928014, P58=0.0006494796091460797, P59=0.0006054518781765665, P60=0.003259744609281181, P61=0.011073188447891862, P62=0.03388407549035496, P63=0.03972707165295319, P64=0.015547111334992382, P65=0.005504706632860309, P66=0.0024384716639336557, P67=0.0025876119206825945, P68=0.002555148724388062, P69=0.005720100394375568, P70=0.011599526525523284, P71=0.014728201669826677, P72=0.023747463353273927, P73=0.04900869380785671, P74=0.05360852683165615, P75=0.028367327207745253, P76=0.020319971841760237, P77=0.028822484910749728, P78=0.03473779890942254, P79=0.03150501432531677, P80=0.01792994854669581, P81=0.024830825673230264, P82=0.046347285697567735, P83=0.04852060707444789, P84=0.05828817355639596, P85=0.08689700779963122, P86=0.05629194758548689, P87=0.05106641303746887, P88=0.10885009469796696, P89=0.07132733832972923, P90=0.022161113809193195, P91=0.00777007130997466, P92=0.005362171376536426, P93=0.024763290204340347, P94=0.10847001090969209, P95=0.07703333863407304, P96=0.02386428527841807, P97=0.016380965545119124, P98=0.010991172559389242, P99=0.006838760385948183, P100=0.009568489846201348, P101=0.01566759733762383, P102=0.007578380517359401, P103=0.0052096948144342405, P104=0.0035729054035261274, P105=0.006588832098173251, P106=0.02190530345334909, P107=0.01315154378303141, P108=0.008958472844896153, P109=0.0046994249661685365, P110=0.002302529749861433, P111=0.0075597303431087556, P112=0.004888559429963361, P113=0.0017947903006039112, P114=0.00038144252599884215, P115=0.000409633512777025, P116=0.0007085292982484357, P117=0.0016120671490727945, P118=0.0012154023332441657, P119=0.0019205470932350016, P120=0.0004938207102500257, P121=0.00048338125109819666, P122=0.001387053680557601, P123=0.0004967704784016605, P124=0.000527229228869712, P125=0.0005184268320275893, P126=0.00025894343639435027, P127=0.0005119694439565338, P128=0.00034157681669207547, P129=0.0002630820131927342, P130=0.0003457823476950008, P131=0.0004960764549685291, P132=0.0008915282605988271, P133=0.0009204946108585846, P134=0.0018630536682582763, P135=0.0016387860578607118, P136=0.0007929322179306167, P137=0.0009651433397223477, P138=0.000621011859148623, P139=0.0006493551690779244, P140=0.0006347049109984087, P141=0.0003415732487702678, P142=0.0005906996222889482, P143=0.00022377064942842839, P144=9.896954424644385e-05, P145=6.117972292512791e-05, P146=6.395184833199827e-05, P147=0.00012756792875093055, P148=0.0001663888494388336, P149=0.0002651543223602556, P150=0.001226291359559011, P151=0.0008662637738612439, P152=0.0003481915792825833, P153=0.0002593084119882213, P154=0.0001303701721585957, P155=0.00010844547131220164, P156=8.268734370811023e-05, P157=8.64577944640138e-05, P158=6.96580808140131e-05, P159=7.810545163654979e-05, P160=0.00010490094671125884, P161=0.00010563582551665342, P162=4.9038538184161245e-05, P163=3.9659554033898736e-05, P164=2.8076742550992052e-05, P165=2.4074197068625402e-05, P166=3.450025221125413e-05, P167=3.096370317793665e-05, P168=4.825977762646647e-05, P169=6.336942695068764e-05, P170=6.0411303098156763e-05, P171=4.028004815951867e-05, P172=4.06020691676455e-05, P173=6.920143228422876e-05, P174=7.310750711022925e-05, P175=5.529677650782522e-05, P176=4.568739830500762e-05, P177=2.828105087132165e-05, P178=1.809179745283934e-05, P179=1.8407526619557163e-06),Row( P0=-731.5105107814618, P1=61.41546004392805, P2=4.781253957474742, P3=3.802943133293237, P4=-2.9769279010428167, P5=1.8321527206301618, P6=5.843460092442156, P7=3.6131044910591164, P8=8.755742152663958, P9=-0.46592898090229007, P10=-2.92921502056346, P11=-0.017893833702058916, P12=3.7561362967343124, P13=-1.9900955439111645, P14=-1.0258922254469125, P15=3.604660723406129, P16=1.9677182530243131, P17=2.335853769525363, P18=1.5519489250872869, P19=2.7544141672370435, P20=-2.4608526089064378, P21=0.41147498462979915, P22=1.1255504385035207, P23=1.4236327869917558, P24=-2.7613431385344636, P25=1.815610593475807, P26=-3.62228100879038, P27=1.2679773421210765, P28=-1.9146642834218643, P29=0.18767877871030086, P30=-2.5413463974741326, P31=5.197733902883221, P32=-2.0477280273698244, P33=2.846340564685817, P34=1.2521840765619772, P35=0.6673169207859077, P36=-0.6752125919142734, P37=-0.042597996579764645, P38=0.3001485163309409, P39=2.248306072001843, P40=0.6950501660421455, P41=0.7025148788944906, P42=0.6978731354935251, P43=0.7566747883540204, P44=0.7866013137401062, P45=0.7703890875329784, P46=0.7858971963965992, P47=0.7949476646007132, P48=0.6807386951612193, P49=0.6773048314641826, P50=0.6985767262009284, P51=0.7051510824971223, P52=2.6810543496216916e-05, P53=9.87379327388312e-06, P54=8.300473905940484e-06, P55=5.291592764462116e-05, P56=0.0006094281994807976, P57=0.0029289361743422807, P58=0.0016782646315944725, P59=0.0013921894453457892, P60=0.0008631024044531034, P61=0.00036152302123295733, P62=0.0007053775523896335, P63=0.0015086070093317092, P64=0.001122233523454231, P65=0.0014557547155188669, P66=0.003045164103510533, P67=0.005250515909614, P68=0.0033607813439860672, P69=0.0016154318868820334, P70=0.0010404070644770259, P71=0.0005037247750229673, P72=0.00020916130935770254, P73=0.00034740653829058574, P74=0.0006822600734334554, P75=0.0011440747652874642, P76=0.0011041609498061065, P77=0.001061648329372448, P78=0.0009977004715038989, P79=0.0012744078475968354, P80=0.0014139576689362315, P81=0.001704411229369619, P82=0.0030141010092143957, P83=0.003137603759838442, P84=0.0022880985867365883, P85=0.003405487619879174, P86=0.0037379696727171396, P87=0.002165603458229156, P88=0.0011552899658787664, P89=0.001409218632348183, P90=0.0041170326454544786, P91=0.005617597846077874, P92=0.0022630271500193202, P93=0.0005025853667679773, P94=0.0007814050879019835, P95=0.004572344075367943, P96=0.004479279181676222, P97=0.003311430635801889, P98=0.0008080301432229467, P99=0.0003127354001073998, P100=0.00035599592771669276, P101=0.0003271884575567875, P102=0.00023989037150530808, P103=0.00010140779028225653, P104=0.00017172612641315414, P105=0.0003790123698051502, P106=0.0006273413761705322, P107=0.0012213489318385156, P108=0.0009189205612486821, P109=0.00030112568199090083, P110=8.132275650550045e-05, P111=0.0003858335591653304, P112=0.00020731758275986556, P113=5.4286516847369856e-05, P114=3.141329719074197e-05, P115=4.6376780996221265e-05, P116=5.79148027063746e-05, P117=0.00010361392655282203, P118=5.314523423775694e-05, P119=4.107741748180658e-05, P120=2.3010077887015876e-05, P121=2.654977326853958e-05, P122=3.41399946235203e-05, P123=3.682906937698495e-05, P124=3.0149471373980903e-05, P125=2.024379243472485e-05, P126=1.0438755139137372e-05, P127=1.3385916995237762e-05, P128=1.1086446228814658e-05, P129=6.194002507385438e-06, P130=5.791634828131171e-06, P131=8.699678792656185e-06, P132=9.777164466564589e-06, P133=1.7784219045166077e-05, P134=4.001952620997026e-05, P135=3.1225866381560156e-05, P136=2.209136304370289e-05, P137=1.6955772470369614e-05, P138=7.600477199574747e-06, P139=7.646473575034894e-06, P140=8.058773495416761e-06, P141=1.0141710008828017e-05, P142=1.7025605298222786e-05, P143=1.5981692280541433e-05, P144=2.549138197252649e-05, P145=1.6389211517128995e-05, P146=6.985923456304546e-06, P147=8.296538066478698e-06, P148=1.2974062597249636e-05, P149=1.7425633539210795e-05, P150=3.3662058844207336e-05, P151=5.408899335122277e-05, P152=5.6507401900311906e-05, P153=8.718283311835628e-05, P154=0.00015545219352372188, P155=0.0001479899014938036, P156=0.0002757710867468457, P157=0.00052006874397591, P158=0.0003629326580534538, P159=0.00020699737792321933, P160=0.00014535262112764686, P161=0.00010728772468531628, P162=5.695226752706621e-05, P163=2.7268498284706672e-05, P164=3.4079797466269944e-05, P165=4.762790969151921e-05, P166=4.718928923609326e-05, P167=3.698266652658184e-05, P168=2.676499264411801e-05, P169=1.6578538196916845e-05, P170=1.3203849331988238e-05, P171=2.2510070300982805e-05, P172=3.0785530608694356e-05, P173=3.4417575673037756e-05, P174=4.993255435038773e-05, P175=5.764380082080178e-05, P176=4.294700945929137e-05, P177=3.1037393633165676e-05, P178=1.2443576099525548e-05, P179=1.0109676731953407e-06),Row( P0=-700.8854890562953, P1=71.81557636836392, P2=-2.409056205906358, P3=9.137424087111574, P4=-5.728687866991659, P5=-7.163494202557118, P6=-2.075364853336537, P7=0.8230885172207247, P8=2.0512514006130087, P9=3.903234882738899, P10=-7.176744502752051, P11=-1.97413961067376, P12=4.519646169901778, P13=-4.017876256578025, P14=0.23646924337035186, P15=2.515766954133142, P16=-1.1098922177850088, P17=1.6303528605042765, P18=-0.3849125179877842, P19=3.0957389281053627, P20=0.32121730439753604, P21=3.5645304225290735, P22=3.908804573752273, P23=3.733151668547374, P24=1.1992696844983164, P25=2.5307426572759537, P26=-0.8711176682391708, P27=3.48014069024148, P28=2.5716804390558563, P29=5.332088249418354, P30=2.034333754496593, P31=3.9809596233634643, P32=-0.9297609195022465, P33=3.8835075211038754, P34=-0.1672504987944363, P35=2.8551224677628633, P36=-0.5653918645738084, P37=1.5449259372612563, P38=0.901790127765602, P39=1.1842977005906414, P40=0.6764156468113113, P41=0.6732226575308786, P42=0.6843965898695235, P43=0.7127363010955252, P44=0.6998589390839, P45=0.7158950486705848, P46=0.7354286457675684, P47=0.7252562948314437, P48=0.7175963551949187, P49=0.6612130905310082, P50=0.6259158754005294, P51=0.6559972577332652, P52=0.00017055404436867874, P53=2.574692370051352e-05, P54=1.2012609008843997e-05, P55=3.022598625719165e-05, P56=6.238486333711202e-05, P57=0.00020914388231596493, P58=0.0007521186263837497, P59=0.0024100621954905228, P60=0.005237843472949094, P61=0.006379607507014684, P62=0.014518396018334441, P63=0.017830257516387067, P64=0.004590002806367166, P65=0.001966529048505943, P66=0.004721587834630192, P67=0.005445307825406439, P68=0.005319039930570939, P69=0.007575989819399229, P70=0.01817342139144685, P71=0.013202042492928669, P72=0.002803433697863693, P73=0.005559409603189161, P74=0.01526747554170931, P75=0.011160755488406688, P76=0.004002401622270216, P77=0.006582653955076226, P78=0.004793524937255061, P79=0.0022253665273292223, P80=0.002756762550101758, P81=0.004136227120491744, P82=0.004423414953200348, P83=0.002408814029066458, P84=0.0029080372177576054, P85=0.004614423934661623, P86=0.004141954907434859, P87=0.0036032582390948488, P88=0.007736766723920283, P89=0.008579677249964482, P90=0.0020046107125731624, P91=0.0005396970978114744, P92=0.004774797522704886, P93=0.004009097545836891, P94=0.0009968169174920068, P95=0.0009116930383217045, P96=0.00238648864559524, P97=0.009315827550300464, P98=0.010551955029570018, P99=0.0015037010475803323, P100=0.0002022595233616751, P101=6.09275414401539e-05, P102=0.00027247888837616965, P103=0.002522257736276208, P104=0.009402639388067485, P105=0.007627699549342483, P106=0.0005879357626883447, P107=0.00068681046950987, P108=0.0013081537195412587, P109=0.003265710544166947, P110=0.004040367591671981, P111=0.002543770065994695, P112=0.0007993624741926256, P113=0.00047211542770851954, P114=0.000912023815179336, P115=0.000345792730838957, P116=0.0005036846409720878, P117=0.0008955768506554204, P118=0.00045593981462910065, P119=0.000509573587137907, P120=0.00044702858768085903, P121=0.0001691365140559595, P122=0.0003541910500135298, P123=0.00011624341272073131, P124=8.543638097780535e-05, P125=8.030300424600618e-05, P126=3.60245207549601e-05, P127=3.1236011561370314e-05, P128=2.470735935963642e-05, P129=2.37871577317618e-05, P130=6.014888293856765e-05, P131=6.426032873647054e-05, P132=8.479810698224077e-05, P133=9.904770497027563e-05, P134=0.0001480952481975696, P135=0.00013174121504793142, P136=0.00017260151104110208, P137=0.00018070510053533806, P138=0.0001422015948888432, P139=0.0001115477789157191, P140=8.434651022717873e-05, P141=8.164047289865522e-05, P142=8.824092736573108e-05, P143=6.75635974577574e-05, P144=5.847184181458896e-05, P145=3.165157257454363e-05, P146=2.41888904117389e-05, P147=8.606990461026405e-06, P148=4.862612467168401e-06, P149=8.707424948403834e-06, P150=1.1475765807398631e-05, P151=1.0040233862908845e-05, P152=1.4810224651588424e-05, P153=2.7450928082183953e-05, P154=1.5183142891730317e-05, P155=1.848295572736521e-05, P156=1.5486665133899547e-05, P157=2.1843076648635085e-05, P158=5.927897278392041e-05, P159=8.593820614954081e-05, P160=0.0001012402116675784, P161=0.00011860474571375304, P162=5.637110619608514e-05, P163=1.6550078975839797e-05, P164=1.0752349007086023e-05, P165=2.1449835740188108e-05, P166=1.8867726083385583e-05, P167=2.0640285329349393e-05, P168=1.0457867186149138e-05, P169=3.0426985740380845e-06, P170=2.5970355239321857e-06, P171=2.7244543353898113e-06, P172=2.4198939746704806e-06, P173=2.870656869748189e-06, P174=3.806085384955764e-06, P175=3.7955198029579697e-06, P176=3.16746307001916e-06, P177=3.142790768510755e-06, P178=1.3352296213630997e-06, P179=1.0987919525093734e-07),Row( P0=-695.3533251826406, P1=58.282999218571156, P2=2.044373204866463, P3=6.890392499615677, P4=-5.799063114329037, P5=-5.115452286513879, P6=-0.4588888176949072, P7=-0.24948202418762658, P8=1.707067477703004, P9=-1.0644730620478868, P10=-1.8074622748245082, P11=1.7030610113223914, P12=2.3763069933583374, P13=-1.5020756287753971, P14=-0.558148983862206, P15=1.0838381347488606, P16=0.713468092225516, P17=1.8956184630136637, P18=-0.9437072588535789, P19=2.7369092323682995, P20=-2.436446297453425, P21=-1.619869936814817, P22=0.6338270632998567, P23=-0.8536362225579703, P24=-3.088265331776065, P25=0.47096391183386266, P26=-1.2650690889465002, P27=2.6029940228287747, P28=1.4696212874744856, P29=4.379553491813658, P30=1.9400543219022517, P31=7.407808840312089, P32=3.4171332979783697, P33=5.126217152916893, P34=2.1892260502540632, P35=2.7489779678477184, P36=-0.051751513990533, P37=1.7623613193783327, P38=1.0957362642919428, P39=1.411544613870116, P40=0.7790877283043599, P41=0.7494422646601289, P42=0.7074198712680574, P43=0.7281643939755303, P44=0.74176024413885, P45=0.7390141957903069, P46=0.72841962004734, P47=0.7404991214101373, P48=0.727925393357589, P49=0.7394877528440664, P50=0.7449400893251712, P51=0.7470779666886026, P52=4.723360625348748e-05, P53=1.3626279531705857e-05, P54=2.2101867191302238e-05, P55=6.396795652661515e-05, P56=0.00017381122656007239, P57=0.0006413460359519651, P58=0.0013216389593576458, P59=0.004446511685133038, P60=0.006183143089177021, P61=0.003846951652657356, P62=0.0010523555554245061, P63=0.00046769879028726326, P64=0.0003549198179218206, P65=0.0014658307618504907, P66=0.006425712402298543, P67=0.016121240678128795, P68=0.01568466884402619, P69=0.014663251546888665, P70=0.016981881101982416, P71=0.008149415217245272, P72=0.0015894058648575527, P73=0.004114553214086486, P74=0.004749284500959415, P75=0.004839997137426021, P76=0.0080221548669749, P77=0.010920219915253923, P78=0.02859895457341746, P79=0.03693336290241137, P80=0.016621025879384202, P81=0.0048335549972423775, P82=0.002969310621915826, P83=0.003692626359402905, P84=0.007632572267348313, P85=0.006847742500482868, P86=0.004063824570187482, P87=0.003608624957684686, P88=0.0043245474185000955, P89=0.004656130985461158, P90=0.002890524846060374, P91=0.002588918211690275, P92=0.013639774062942119, P93=0.0062634315511117195, P94=0.0030552952253117467, P95=0.0036643351059676473, P96=0.0023590909803323104, P97=0.0024581778723300625, P98=0.004345177578607648, P99=0.0016645406922663186, P100=0.0006952484881434061, P101=0.0006810376711939981, P102=0.0005700449844173412, P103=0.00037527102479538416, P104=0.0002325135599617218, P105=0.0002729255833715228, P106=0.0007469234799074088, P107=0.0012545648159273573, P108=0.0007416172030581051, P109=0.0003256823434859582, P110=0.0006446904515128626, P111=0.0016601686660700577, P112=0.0004388924025606157, P113=7.928250738268467e-05, P114=0.00013333953340215042, P115=0.00013579150652255682, P116=7.542871305300425e-05, P117=0.00028725462959121323, P118=0.000291545275933121, P119=0.0002139760039599923, P120=0.00022228375462409635, P121=0.00044213702813932713, P122=0.0001580328106284601, P123=0.0001191724591069018, P124=5.447526687619068e-05, P125=6.14472539941978e-05, P126=6.254648084332012e-05, P127=6.014678526911649e-05, P128=4.779597915194269e-05, P129=3.5402669089531844e-05, P130=4.7522629291393134e-05, P131=5.28397144467971e-05, P132=6.457379172844317e-05, P133=8.834608596275602e-05, P134=6.629597791648241e-05, P135=3.505193906180629e-05, P136=3.32295284384297e-05, P137=2.0051346497200295e-05, P138=1.2062303718031889e-05, P139=1.9003343214881606e-05, P140=1.947165025614098e-05, P141=3.8871143645908476e-05, P142=5.4519079925772615e-05, P143=4.812284450774045e-05, P144=3.0882510164142695e-05, P145=3.1859374057146254e-05, P146=4.002470515355331e-05, P147=2.35367705822931e-05, P148=2.4633993944892404e-05, P149=2.097475245429094e-05, P150=3.999031145741183e-05, P151=5.73839735839783e-05, P152=0.0001386908412164696, P153=0.00039235776950357585, P154=0.00042165726768339886, P155=0.0002710135896157073, P156=0.00044104075849870034, P157=0.0004713703080179567, P158=0.00035492447262220825, P159=0.00020449476036557684, P160=0.00010219077869193916, P161=6.0034644728556636e-05, P162=6.507385344672005e-05, P163=5.694119269812283e-05, P164=5.791984063991342e-05, P165=3.4883758855175724e-05, P166=3.0515674638473715e-05, P167=2.086436132376316e-05, P168=3.06474979455834e-05, P169=2.600854693177131e-05, P170=2.3210790702949703e-05, P171=8.753701655189934e-06, P172=1.1250579176754302e-05, P173=1.8768510790990133e-05, P174=2.108118800382627e-05, P175=2.3731377610579294e-05, P176=1.9290528165471574e-05, P177=1.4847469029681146e-05, P178=6.40375157161184e-06, P179=5.612857203246043e-07),Row( P0=-566.4285919244356, P1=51.38933028805871, P2=-24.120450416502738, P3=-1.9052623995589912, P4=-10.18427684967979, P5=-11.536088620778978, P6=-5.586547933120288, P7=0.9952551662612055, P8=-3.20770460356302, P9=1.3428146063676756, P10=-9.053296384617225, P11=-4.213359888729865, P12=6.086795952748075, P13=0.07388371885719912, P14=10.684801176035936, P15=13.25449943365812, P16=7.188439483590467, P17=11.405713187358169, P18=6.461579446183881, P19=7.292788482056136, P20=2.8706159770933852, P21=3.2804438416216426, P22=0.5375479558570843, P23=1.795569979735406, P24=-1.1094243686331788, P25=4.364426891481389, P26=0.6436178254635082, P27=3.3936067495525255, P28=1.4926075007278417, P29=1.145937415265321, P30=1.0722411530228493, P31=2.7034110911773714, P32=-1.1595016853410447, P33=3.94761387471713, P34=-0.4153148723630431, P35=1.8804546590600757, P36=-0.5026348968221408, P37=1.1082105054999865, P38=3.711200701725859, P39=5.223559678982128, P40=0.658868789821293, P41=0.660274420703536, P42=0.6476681541929925, P43=0.6768317646147082, P44=0.7140266671653714, P45=0.6709507919929166, P46=0.6046389950881766, P47=0.621676538363868, P48=0.6266529205469656, P49=0.6389575186044036, P50=0.6707764942876069, P51=0.6629270571616755, P52=0.022087691440988343, P53=0.0015177674036450175, P54=5.6435331765354e-05, P55=8.546048922775063e-05, P56=0.0002478727598844036, P57=0.0009280509270591593, P58=0.0006023524159611963, P59=0.0003333291461420998, P60=0.00041895306701810934, P61=0.0007279400442363745, P62=0.0031547285796334774, P63=0.010962471465931089, P64=0.020438666878087965, P65=0.05423236555532386, P66=0.0642237303626371, P67=0.09207190351766266, P68=0.10789082083639828, P69=0.11345303648670818, P70=0.13413121588208482, P71=0.09953883443411075, P72=0.037811957249520134, P73=0.01181358713304201, P74=0.0038428301791356523, P75=0.002078787008601435, P76=0.0018604059744578522, P77=0.0045337238144094755, P78=0.016112117448859237, P79=0.03692434951997228, P80=0.03477857266624923, P81=0.06650851630884894, P82=0.14117803134865842, P83=0.18223709583000539, P84=0.17625087361719305, P85=0.22801824268389417, P86=0.12448573753799415, P87=0.07328845722634274, P88=0.1007346440578514, P89=0.08646657132152721, P90=0.1042185631607595, P91=0.13945010191647683, P92=0.19131124329202231, P93=0.08007523794042021, P94=0.01918277926592286, P95=0.018219332615857155, P96=0.009979398798760064, P97=0.012490611012734312, P98=0.025813454157599055, P99=0.02973465939955284, P100=0.05948295478552198, P101=0.045997874027493606, P102=0.03278625272352741, P103=0.07225501481935155, P104=0.017252400428192177, P105=0.0063366295500716634, P106=0.028143170057427787, P107=0.06943876821282546, P108=0.018601296100463245, P109=0.003951627617873026, P110=0.012470064961413478, P111=0.04991093502508478, P112=0.07228952406225043, P113=0.05668785454492561, P114=0.013140984063386447, P115=0.0026717408751870853, P116=0.003944727269440703, P117=0.022145154620635172, P118=0.01963760324817051, P119=0.015940333725830768, P120=0.003464252374445175, P121=0.0057164764606420304, P122=0.013450710983063763, P123=0.003992437722365798, P124=0.004910891729362157, P125=0.0027246098428138766, P126=0.0033485362858994725, P127=0.004171229916195331, P128=0.0026482124766292322, P129=0.00336425576918267, P130=0.002342616259797779, P131=0.0021854499427953603, P132=0.0020947987996245325, P133=0.002030107747015537, P134=0.00228840451875615, P135=0.0026959493330039183, P136=0.004311190940682047, P137=0.0036037667509103886, P138=0.002102065051574776, P139=0.0016423128294809513, P140=0.0012608369640233709, P141=0.001262034928978298, P142=0.0014516170011794338, P143=0.0015471759120314827, P144=0.0014875786733545937, P145=0.0006631976120095125, P146=0.00025374133490809055, P147=0.00012637508862882706, P148=6.857243799587375e-05, P149=8.396144624338936e-05, P150=0.00029662012381709486, P151=0.0006338604698504605, P152=0.0010168039572365755, P153=0.0026963254805622817, P154=0.0010697245051064328, P155=0.0008477243751313886, P156=0.0005875726777558619, P157=0.00046207269558463037, P158=0.00048709204077331496, P159=0.0005277157761017346, P160=0.0005968548377610099, P161=0.0005266304230995449, P162=0.00039487328289006403, P163=0.00036524630685278147, P164=0.0002779902119249456, P165=0.0001864828511295062, P166=0.00016292877197037302, P167=0.0001499947200693942, P168=0.00016984747712284434, P169=0.00012664999264202858, P170=7.102281054343353e-05, P171=6.501002222281745e-05, P172=9.804143448593213e-05, P173=0.00013709865415223983, P174=0.0001907263098232293, P175=0.00017671214676344405, P176=0.00012988907821607837, P177=8.551172755463579e-05, P178=2.7171619827564354e-05, P179=3.938764896274876e-06),Row( P0=-679.9684802535136, P1=52.720940082257776, P2=2.3672592763055476, P3=0.7712311068068273, P4=-7.061073989306683, P5=-1.0193455435336893, P6=0.8229390505634561, P7=0.2718450598419263, P8=4.265700448731421, P9=-2.067151993290677, P10=-3.1609840771088256, P11=-0.5328400379690618, P12=0.7901451029108695, P13=-2.5264741629939764, P14=-1.3702984132878684, P15=1.8343400705875232, P16=2.754887383184402, P17=1.0993292430661943, P18=1.040827613973232, P19=0.9809342931340687, P20=-2.8281103739561297, P21=-0.9934061766266691, P22=-0.9130160941524471, P23=2.2367996546848894, P24=-1.4795312077119038, P25=1.537035388600979, P26=0.18952798933545467, P27=2.444558461373534, P28=0.7159898615690281, P29=1.6055355357667256, P30=-2.460361054699865, P31=3.92653260530692, P32=-1.8911548811979022, P33=0.7618953844536208, P34=0.16174598239485893, P35=0.3085162428120084, P36=-0.684721527873504, P37=1.6895931224144658, P38=1.593204272191368, P39=2.4218131003931385, P40=0.7232290561886885, P41=0.7288587356388819, P42=0.7191033849004479, P43=0.7582295440212369, P44=0.7703644580611001, P45=0.780649307963992, P46=0.7756178671052453, P47=0.8147112194812032, P48=0.7718582641141433, P49=0.7347271782526418, P50=0.740617083127214, P51=0.7119580455508693, P52=6.492973230855837e-05, P53=1.6081262560017565e-05, P54=7.060830064979028e-06, P55=6.0570834230754115e-05, P56=0.0005720694740174354, P57=0.003216539771096872, P58=0.002586517356200974, P59=0.0014713559465784906, P60=0.00216306709878476, P61=0.0048468559898167595, P62=0.006465933453351538, P63=0.004600811337856333, P64=0.0023004814379499, P65=0.0033873438349009727, P66=0.005434559226758932, P67=0.004922180814401275, P68=0.0036589008753629883, P69=0.0048571388964676315, P70=0.010589165645779533, P71=0.009843796473691475, P72=0.007480236871546438, P73=0.01017654570624231, P74=0.008005709297038413, P75=0.00424721170608334, P76=0.0027004533851147715, P77=0.002709451211007957, P78=0.006670726000964626, P79=0.01583212558704399, P80=0.025320115642766206, P81=0.02675347023601053, P82=0.021700318765881004, P83=0.01612218206988808, P84=0.02543034283873164, P85=0.03940752660773662, P86=0.02154118364021599, P87=0.006786447246159839, P88=0.004203254429513974, P89=0.007565723133207683, P90=0.01557394248545161, P91=0.020573127832641338, P92=0.014908087659958564, P93=0.006512232902681148, P94=0.0039996021519466355, P95=0.005779659697186795, P96=0.008393892234663962, P97=0.009449734621427633, P98=0.006039489983807186, P99=0.0018311213360893465, P100=0.0013088462407216575, P101=0.002791713693050117, P102=0.0033240266632794655, P103=0.0025574400419882165, P104=0.0009352072918946738, P105=0.0008381604624967967, P106=0.004372659450540909, P107=0.01178648559584072, P108=0.006167271627930869, P109=0.0008305643038095886, P110=0.0002444058030577937, P111=0.00032480966725970833, P112=0.00046413595462878305, P113=0.0003318297628757177, P114=0.00015707747546343464, P115=0.0001851895493400079, P116=0.00018409098701384837, P117=0.0003861153626849285, P118=0.0012315226952929006, P119=0.0004791860718652976, P120=0.00018530488512565377, P121=0.0001993702147881459, P122=0.00022298352598516645, P123=8.372731877495922e-05, P124=4.660893400991419e-05, P125=4.7040154378475596e-05, P126=3.192490735398003e-05, P127=5.5347546079666475e-05, P128=3.646004571455763e-05, P129=3.198998173167571e-05, P130=3.139720530696025e-05, P131=2.7814891456463882e-05, P132=3.8866320303021164e-05, P133=8.35214706739278e-05, P134=0.00020250874173437537, P135=8.142432771859915e-05, P136=5.7054360361253146e-05, P137=4.56047598804617e-05, P138=4.899952611015291e-05, P139=0.0001035116638949808, P140=0.00010973882211879284, P141=0.00011804671054491817, P142=9.072220990103616e-05, P143=6.045467058914941e-05, P144=6.207206450595843e-05, P145=2.587842794840323e-05, P146=2.678446092225725e-05, P147=2.6269618372037026e-05, P148=3.9039015293391116e-05, P149=5.278819790082829e-05, P150=8.840006251507602e-05, P151=0.000101663966380847, P152=9.04148201429355e-05, P153=0.00011507359787365593, P154=0.00011372117990506406, P155=0.00010985375072881442, P156=0.00014743418967305455, P157=0.00027624898314788763, P158=0.0003418404285841248, P159=0.0003988161259178948, P160=0.00029044013803859, P161=0.0003473757457047094, P162=0.00014407736987663162, P163=6.820450356489836e-05, P164=5.341688339771967e-05, P165=7.158227145870288e-05, P166=8.698897459848156e-05, P167=8.03390716823796e-05, P168=6.467303752390218e-05, P169=3.852163086642699e-05, P170=1.7501121924749225e-05, P171=1.732927400146033e-05, P172=2.488640833916973e-05, P173=3.6442635720072525e-05, P174=6.65589047688928e-05, P175=5.4607630538009915e-05, P176=5.868921500470925e-05, P177=4.17128956320599e-05, P178=1.2352421979712979e-05, P179=1.4283849422270006e-06),Row( P0=-633.1138226519871, P1=69.28648368385815, P2=-11.505175572500134, P3=1.9215704331762196, P4=-11.569086508592767, P5=-7.751983505255544, P6=-2.5045420088274186, P7=-0.7389790219425953, P8=0.059691267373230675, P9=-1.945523893640534, P10=-7.675258934218906, P11=-1.7731303738718176, P12=-0.5077639104150845, P13=-5.74096197054788, P14=-1.7179162554539607, P15=-1.1845513889187713, P16=-1.544224140411416, P17=-0.06953522378570666, P18=-1.4784432318511695, P19=2.417986690823184, P20=-1.0082385742242375, P21=4.3633101486625225, P22=8.005333290012151, P23=5.49694155971429, P24=5.471948130753348, P25=6.069966808339649, P26=1.7449594306248422, P27=4.5645119347808025, P28=-0.5381109452336178, P29=0.4739544442549399, P30=-1.5544145617385545, P31=1.272978267567834, P32=-2.03934606561676, P33=1.1965776946332596, P34=-3.188389947256499, P35=0.018185393057834624, P36=-2.311263036983934, P37=1.484628937962867, P38=1.4736166183782704, P39=1.8892154886039225, P40=0.6340397304729901, P41=0.6194705997854415, P42=0.6399458253401987, P43=0.6853425043829591, P44=0.733102374783971, P45=0.7569510681555501, P46=0.7618883188577809, P47=0.7470425650322606, P48=0.7190104586502479, P49=0.6807868384612573, P50=0.6446494305173156, P51=0.6377244203445627, P52=4.6588965227733944e-05, P53=1.834176631410055e-05, P54=2.155747608344749e-05, P55=9.270126750031276e-05, P56=0.0007362172601062131, P57=0.0038829099177206347, P58=0.0022457392706643855, P59=0.001026371643202042, P60=0.004236988754409403, P61=0.01919264532970911, P62=0.030970646670483336, P63=0.03298893083764296, P64=0.019217940071302, P65=0.00801004646184845, P66=0.0026989641871079176, P67=0.0033166441275886663, P68=0.0031634157544900877, P69=0.004411105254885888, P70=0.007708089433924726, P71=0.01893322283751142, P72=0.02281009407085217, P73=0.022027161443833793, P74=0.020483724748881264, P75=0.02161377594065831, P76=0.02136770086406675, P77=0.03628709589368617, P78=0.05588830073615398, P79=0.03488066251610891, P80=0.019447070484142036, P81=0.03329732746913468, P82=0.0538248490617898, P83=0.05678177899343012, P84=0.06352199328345691, P85=0.0531612007768775, P86=0.023343144684846686, P87=0.02140214219706187, P88=0.02094817152035849, P89=0.01570745277049263, P90=0.04525222384470289, P91=0.03493154574030974, P92=0.028416442174065245, P93=0.02373020612474201, P94=0.017095620870426115, P95=0.015652936194028588, P96=0.016400761926168256, P97=0.013070625568500997, P98=0.011876448219572925, P99=0.015959550565468994, P100=0.007864587077087749, P101=0.005090455532994954, P102=0.002487083748291938, P103=0.0019742881054880153, P104=0.0020245525172881028, P105=0.0027267353396525458, P106=0.005077964782571505, P107=0.007174146564232871, P108=0.0077656549503908375, P109=0.0038969792698121865, P110=0.0024737884618379084, P111=0.00972854472980626, P112=0.00363094441756638, P113=0.0016638189155944471, P114=0.0014008305884527033, P115=0.0009444656902767543, P116=0.0005022218980376174, P117=0.0009627114458140093, P118=0.0025085522324819172, P119=0.0021183357123777916, P120=0.0005554815867704508, P121=0.0007626686263311537, P122=0.0009627918344304228, P123=0.0010813348818250342, P124=0.0010400781368091118, P125=0.0005618138330224465, P126=0.0004040328267174777, P127=0.0003594160070781052, P128=0.00022437154982796686, P129=0.00011962972518921836, P130=0.0002687501522770195, P131=0.00015646049236910046, P132=0.00017781356005164403, P133=0.0002804917376996632, P134=0.0005170934967188129, P135=0.0003832098038076098, P136=0.0004865091187697019, P137=0.0004309787295511071, P138=0.00015264617387485724, P139=0.00020762536382628064, P140=0.00013592396262952091, P141=0.00017397986766829314, P142=0.00023522056410596952, P143=0.0001835825033285383, P144=0.00035782069354667486, P145=0.0003297678826513359, P146=0.0002467926263703824, P147=0.00010320636885825191, P148=3.1769940354581915e-05, P149=1.6298642963909347e-05, P150=1.4988672378321724e-05, P151=2.5301690062547997e-05, P152=6.359679101123105e-05, P153=0.00014499192370018884, P154=0.0003021996662728711, P155=0.00045767054725009406, P156=0.0002985419499222326, P157=0.0004178572730772825, P158=0.0005413378600883839, P159=0.0006397464118079354, P160=0.0005881173578052377, P161=0.000439388601824555, P162=0.00031334310456237915, P163=0.00011918333608594262, P164=0.00020335605347733895, P165=0.0002006687834678893, P166=0.00012985910002549263, P167=6.314132347675552e-05, P168=4.9920465866343525e-05, P169=3.2654130668827446e-05, P170=2.093909831574794e-05, P171=1.2717881256192195e-05, P172=1.1189043055155799e-05, P173=1.555929609205508e-05, P174=1.7735705652678284e-05, P175=1.7554146841053824e-05, P176=1.3813100664547132e-05, P177=1.117674399271125e-05, P178=4.320102556764398e-06, P179=4.598594116828111e-07),Row( P0=-748.5190968949714, P1=81.53913260479369, P2=1.9520069762067458, P3=10.544727143414125, P4=-5.38436486886331, P5=2.081367110812538, P6=4.668038319759498, P7=4.88480407436458, P8=1.39209595246198, P9=-0.38212701374325514, P10=0.0073398424984156396, P11=1.924593815280131, P12=1.3699567811809783, P13=0.44537899189031077, P14=-2.6121679299187126, P15=2.2143253409079695, P16=1.7329886621514388, P17=-0.09165763971963327, P18=1.5778120110536233, P19=3.1147593307553825, P20=-3.839571861780064, P21=1.2737650290369666, P22=-1.6769592721011093, P23=-0.3162618171013699, P24=-2.0091372517616124, P25=-2.1282877117905663, P26=-1.501898873001406, P27=1.8610115459397132, P28=-0.600628415440092, P29=0.6699321391093719, P30=1.201645924311758, P31=5.5155276897479215, P32=2.8703178590832783, P33=5.852405160240125, P34=1.5599569278639587, P35=2.4072745401884252, P36=1.7229514029780264, P37=2.6254722317137404, P38=2.435264543672196, P39=3.1521496028204434, P40=0.7504404846469033, P41=0.7200503106004501, P42=0.6812138280801202, P43=0.6733896900551011, P44=0.7064647158982389, P45=0.700550129277475, P46=0.7109862648520427, P47=0.7459055259376144, P48=0.7451733320626, P49=0.7624997521490275, P50=0.7133587234398651, P51=0.7195286700156924, P52=0.00019268178097560577, P53=3.2596832704394436e-05, P54=1.022887442914374e-05, P55=4.377722716197255e-05, P56=0.0001650917799402064, P57=0.0010941247515791634, P58=0.002262309173375823, P59=0.004193087377764211, P60=0.0025552594604017274, P61=0.0006299396018946474, P62=0.0002647579830650067, P63=0.000243638406296392, P64=0.0005244766844190076, P65=0.0010335939284428042, P66=0.002614003583317137, P67=0.00656704301045187, P68=0.005321445637827453, P69=0.005217041243867743, P70=0.0018099284103683647, P71=0.0005556114168479717, P72=0.0003879046471335465, P73=0.0002742220261751664, P74=0.000349533408733379, P75=0.0007954582385547125, P76=0.0010790411384456805, P77=0.00634476836734168, P78=0.007847560009294416, P79=0.002873578081355256, P80=0.0006546359618493935, P81=0.00021946348612713118, P82=0.00037328354719545206, P83=0.0008747742609818373, P84=0.0008915010361429422, P85=0.0007398698569133009, P86=0.0006669502761081146, P87=0.0002684304878093969, P88=0.00015683491179784533, P89=0.00046542013394761247, P90=0.0013547289510057628, P91=0.0011674432353875914, P92=0.0007448770193155609, P93=0.00026768154523401123, P94=0.00034284506448870885, P95=0.001172059699226237, P96=0.0012014104436430723, P97=0.00032563897404198634, P98=0.00017952702513657, P99=0.0002050739302952199, P100=0.0015775991027593647, P101=0.0007521070772414706, P102=6.222353554977724e-05, P103=6.206714806245562e-05, P104=0.00048328737129677083, P105=0.0003182346612404838, P106=0.00012037298498352591, P107=0.0002050563826267151, P108=0.0006225259388235142, P109=0.00012034505909752147, P110=3.055864659685887e-05, P111=0.0002101691731386581, P112=0.00010150900757448936, P113=1.535358738539484e-05, P114=2.7827134382724525e-05, P115=3.1789096946848766e-05, P116=1.3368741008266913e-05, P117=4.1386282352428784e-05, P118=2.0930942379447184e-05, P119=1.5253484843923449e-05, P120=1.7874775292098453e-05, P121=1.4841326384407273e-05, P122=1.3227555778571742e-05, P123=8.011008005919912e-06, P124=1.621662974441955e-05, P125=6.90777044327054e-06, P126=2.004364402054735e-05, P127=2.1351425525992067e-05, P128=2.1613594678780043e-05, P129=2.2783257027451638e-05, P130=2.075156654368074e-05, P131=1.3260307185743424e-05, P132=8.198393904006982e-06, P133=6.74445658112243e-06, P134=1.2071348138142726e-05, P135=1.3266904594625976e-05, P136=1.7890351394568035e-05, P137=1.436804288666989e-05, P138=1.0630749323133019e-05, P139=6.3757553385479655e-06, P140=2.963880784051681e-06, P141=2.048450597183088e-06, P142=1.7945635407193144e-06, P143=1.4601047555943778e-06, P144=2.9033281544128833e-06, P145=4.607099888586583e-06, P146=5.3603286962535695e-06, P147=4.668889098882227e-06, P148=3.1699935034341272e-06, P149=4.550617038794023e-06, P150=7.433665317347566e-06, P151=8.425800174303036e-06, P152=9.869979706240785e-06, P153=6.321960433964403e-06, P154=3.236729628825941e-06, P155=6.567281736172274e-06, P156=8.43211739059607e-06, P157=1.3200137721548372e-05, P158=2.3224620466618646e-05, P159=3.101679370257078e-05, P160=1.6452858228451187e-05, P161=1.2824496840218882e-05, P162=4.937276333967803e-06, P163=1.6706776842485041e-06, P164=1.7347945801411296e-06, P165=1.4516890418803406e-06, P166=2.249444444172959e-06, P167=1.660223319043234e-06, P168=8.708559845077515e-07, P169=8.626040170173445e-07, P170=6.395015142145714e-07, P171=6.069674760432588e-07, P172=9.271887721751189e-07, P173=9.590677957877049e-07, P174=6.351113929297597e-07, P175=4.555392485658118e-07, P176=4.1131841185709047e-07, P177=2.8527240884722153e-07, P178=7.417132533681411e-08, P179=8.490829787391207e-09),Row( P0=-724.0050096735092, P1=70.44703645815851, P2=2.167740299783405, P3=2.858245915302314, P4=-5.941772019953589, P5=2.779330079747968, P6=-0.19972903339590267, P7=5.13452943346958, P8=1.8753968417179596, P9=-1.9238052442700107, P10=-2.3772461409605703, P11=3.049149083197032, P12=1.3604274935324676, P13=-1.4407744402110017, P14=-1.1416628808373133, P15=1.7720891050577625, P16=2.3837874181246828, P17=0.25380444368023136, P18=0.18484485059139136, P19=1.6489904585427726, P20=-4.342653958547119, P21=1.1985368298845742, P22=-1.4821749600310425, P23=-1.2595063400673943, P24=-2.96379535948516, P25=-1.0728820690682608, P26=-3.4308708009123148, P27=3.6911632808504065, P28=0.7280616464489464, P29=6.152940953546765, P30=4.178067206057305, P31=10.233372774156884, P32=5.199686000586703, P33=5.384783128715597, P34=0.5586602560849377, P35=0.0017608274336357197, P36=0.246357105203633, P37=2.7345054964826283, P38=2.214301535966198, P39=3.4350730479901435, P40=0.7057987797947656, P41=0.6826548429980113, P42=0.6394617706872864, P43=0.6410547328258902, P44=0.6892132500595369, P45=0.6885919611226281, P46=0.6773293897130885, P47=0.6931287046525674, P48=0.6858650585232312, P49=0.6414317407584481, P50=0.6433461834758774, P51=0.6844466304708641, P52=6.235029559650245e-05, P53=1.174524262864516e-05, P54=4.150893371977378e-06, P55=2.8991546268014734e-05, P56=0.0008041854167567716, P57=0.003654854877963235, P58=0.002258921465924119, P59=0.005559635270108686, P60=0.008145350351552193, P61=0.0014538291429506064, P62=0.00038577152615325576, P63=0.000677254023643162, P64=0.0006504128226919679, P65=0.0005980754341316849, P66=0.0016783705166799193, P67=0.004355113647459495, P68=0.011506299621617522, P69=0.010273548329858582, P70=0.002700401157317873, P71=0.00060337535057626, P72=0.0002577616846489037, P73=0.0006669146719135739, P74=0.0014108865071393376, P75=0.0018406044271607367, P76=0.0028796362604546176, P77=0.006208179218951952, P78=0.005162619035356495, P79=0.002679693027793222, P80=0.0008646868179603923, P81=0.0005292790370885694, P82=0.0010124015303700612, P83=0.002495918216932358, P84=0.00877632655714013, P85=0.015584724570432841, P86=0.009891408209678718, P87=0.0023490933256211884, P88=0.00034753839366574133, P89=0.0003576938019991291, P90=0.0009242521740843464, P91=0.006443952448764611, P92=0.011785963783212552, P93=0.008171183919812166, P94=0.0004843616314847275, P95=0.0005753698323998247, P96=0.0005829501534158324, P97=0.0025605078678758576, P98=0.006237511335863636, P99=0.002510416092125241, P100=0.00015506361417269522, P101=0.000294407259696932, P102=0.0007574430973831787, P103=0.0010357047365838674, P104=0.00018511249723037536, P105=0.0001037380847335709, P106=0.0008066961270053492, P107=0.0023333049183545764, P108=0.0005751210567005104, P109=0.0001159875249734892, P110=0.0003256841871956382, P111=0.0009186812154710026, P112=0.00014557760234430833, P113=3.204157312966761e-05, P114=6.963593273005977e-05, P115=7.073371263974218e-05, P116=4.040012212004554e-05, P117=9.86946696741844e-05, P118=0.00013381097242684689, P119=4.933059570325307e-05, P120=3.407168194962656e-05, P121=5.2587502126695946e-05, P122=3.1327624643059425e-05, P123=6.201954214430469e-05, P124=4.446838568769235e-05, P125=1.5355064979853368e-05, P126=1.333981930994683e-05, P127=1.2134597351918468e-05, P128=7.799933154377253e-06, P129=5.458894260265569e-06, P130=6.410594155630133e-06, P131=6.171342924631061e-06, P132=4.77919353222043e-06, P133=7.743503769308224e-06, P134=1.5561451248620747e-05, P135=1.4140349361588187e-05, P136=1.4480328885288193e-05, P137=9.826889114479438e-06, P138=5.379311396911682e-06, P139=4.685224820762116e-06, P140=3.927432107442631e-06, P141=4.835193917658754e-06, P142=7.019129369415656e-06, P143=7.513866892887435e-06, P144=9.536391374795323e-06, P145=6.859816061496146e-06, P146=9.94800086129438e-06, P147=6.294146832306138e-06, P148=8.546825528355718e-06, P149=8.353302854595175e-06, P150=1.2699391403344981e-05, P151=1.8638553771711508e-05, P152=2.0345899061937006e-05, P153=3.549304847301655e-05, P154=5.6591398323844083e-05, P155=9.768379821290524e-05, P156=0.0001584809035238371, P157=0.0001671059225148946, P158=0.0001622919376795818, P159=0.00010323156491576215, P160=5.438490212452121e-05, P161=2.4788405755245755e-05, P162=1.7429100568260315e-05, P163=1.5118602409342927e-05, P164=1.548802109188084e-05, P165=1.4111520028390427e-05, P166=1.2006023986470084e-05, P167=7.547679555338207e-06, P168=4.302241089062453e-06, P169=2.15618587880712e-06, P170=3.0665918221012253e-06, P171=3.7630116736344418e-06, P172=5.244040817019332e-06, P173=5.611105535724135e-06, P174=4.901164527985851e-06, P175=3.525609246482246e-06, P176=2.9092807925717646e-06, P177=1.5466593830480398e-06, P178=5.749909476113847e-07, P179=6.453576101058979e-08),Row( P0=-597.0368978969781, P1=52.07843715773347, P2=-9.872939852355888, P3=-10.757765831459054, P4=-17.210427439803713, P5=-11.946303032820582, P6=-4.901782925720153, P7=0.00417250378371145, P8=-0.012506262659580398, P9=-4.318139059794164, P10=-2.982038278971686, P11=-3.416144923875122, P12=-3.0062661962525854, P13=-1.8767917293701717, P14=-2.805799109805117, P15=-1.614604555169052, P16=-1.3665639310738158, P17=1.7115197652880456, P18=2.482094504458555, P19=5.411375410817595, P20=5.0746179646848555, P21=3.8268621129104288, P22=4.436616728318152, P23=3.617354260411955, P24=1.2414866115777865, P25=2.7061463867108193, P26=0.3251027893032869, P27=3.0450412912891904, P28=1.7017821806797688, P29=1.756262468755225, P30=-3.150234110079921, P31=4.851916763980181, P32=-1.7712136264679241, P33=6.446169314991259, P34=-0.5982411863892423, P35=1.8140929599115236, P36=-1.1666498201768305, P37=2.6175652729956953, P38=2.475221949106111, P39=3.392289494832732, P40=0.7420226334630449, P41=0.6959889857343885, P42=0.6788549874409072, P43=0.7297743035729853, P44=0.7449334579793608, P45=0.6911464454915952, P46=0.6867699825228102, P47=0.7148741844259895, P48=0.7242485913880453, P49=0.6924078615514426, P50=0.670851779154848, P51=0.6935942390253925, P52=6.298338705940261e-05, P53=9.040513112096637e-06, P54=7.366604650754728e-06, P55=1.946514058422542e-05, P56=6.796521471783793e-05, P57=0.00033309823599032547, P58=0.0004460624291574871, P59=0.000805126941876055, P60=0.0014868164519005497, P61=0.004022124524254334, P62=0.007718604345703056, P63=0.009996903866458604, P64=0.011164247445073605, P65=0.013045243327184418, P66=0.005187086734444629, P67=0.0028609602701718585, P68=0.002344222933312021, P69=0.002012833372625271, P70=0.0037512353872725824, P71=0.005228185319331361, P72=0.006217183542259021, P73=0.009252725126402484, P74=0.011394586991341183, P75=0.015060529742549974, P76=0.024150154837972254, P77=0.07421828793080905, P78=0.16626577364740874, P79=0.2603898111368767, P80=0.19686401334238784, P81=0.07423946749566453, P82=0.04475863274039174, P83=0.04645379119099411, P84=0.05221185204487172, P85=0.04166734506109135, P86=0.024819971998254437, P87=0.019776439092254364, P88=0.03087628584956654, P89=0.033882445750360916, P90=0.03935365237656705, P91=0.05800723678577996, P92=0.016090965777161993, P93=0.004001467562488647, P94=0.00426470216953851, P95=0.006245942169987432, P96=0.00544881506815305, P97=0.003482546568073335, P98=0.004106927993001707, P99=0.005490952854032662, P100=0.003622112664795942, P101=0.001049196924755782, P102=0.0006463070534417171, P103=0.0006507881911518419, P104=0.0011682701576233377, P105=0.0036956553138078503, P106=0.014422457114057523, P107=0.014563236695207513, P108=0.001556452839532725, P109=0.000690113715392652, P110=0.00031195493039960174, P111=0.0008145014613682621, P112=0.00043856407846512975, P113=0.0007511700792608503, P114=0.00027559346209053535, P115=0.00010821071450124255, P116=0.00019940692667285066, P117=0.0003096556539662703, P118=0.000322349800298764, P119=0.00030897273001613003, P120=0.00011341896282597284, P121=0.00025919587214256535, P122=0.0009444081752122934, P123=0.0008057200060511298, P124=0.0004558619141122599, P125=0.000422846421232377, P126=0.0009396167355231042, P127=0.0005802287580236793, P128=0.00024084789515173272, P129=0.0002635109630522721, P130=0.00016084311142070651, P131=0.00014891218402026933, P132=0.00015854804393658687, P133=0.0001267922920798388, P134=0.00021269261131676316, P135=0.0002042319902657299, P136=7.86172601064397e-05, P137=5.824750277207654e-05, P138=2.4547404261996787e-05, P139=2.8433415020776297e-05, P140=2.8152787365932897e-05, P141=3.842302513392133e-05, P142=5.817553422145668e-05, P143=4.223575194758097e-05, P144=6.882243541719804e-05, P145=7.719300355018524e-05, P146=6.558451781815966e-05, P147=2.4900967441166886e-05, P148=2.6188384052057745e-05, P149=4.5582619792313004e-05, P150=0.00010465908075013285, P151=0.00015737963922600587, P152=0.00021927743794092127, P153=0.00028357863592561836, P154=0.0002237958433683117, P155=0.00019280781464769815, P156=0.00027161442487867477, P157=0.0004887203120101198, P158=0.0006170996142812559, P159=0.0006480963063630261, P160=0.0007117872578162675, P161=0.0007375741520153827, P162=0.0005074979809641635, P163=0.00023876646207800272, P164=0.00010663532849873577, P165=0.0001982257645420342, P166=0.00034896193936146297, P167=0.000422957174665375, P168=0.0004968353962420177, P169=0.00040709512159916656, P170=0.00023636265107048024, P171=6.842314861496782e-05, P172=1.9402957169935405e-05, P173=2.512232364535864e-05, P174=5.7254308265740595e-05, P175=8.651208204188925e-05, P176=0.00011885461249196093, P177=0.00011767929943821304, P178=4.533583791788919e-05, P179=4.331619614442718e-06),Row( P0=-763.5320973333996, P1=79.89973658474085, P2=5.462063191031383, P3=10.11734279792346, P4=-5.588163430157302, P5=4.386338996915915, P6=2.5701314174704306, P7=5.9748982545177345, P8=3.602733164242137, P9=-1.9862118209418502, P10=-3.0230914484481377, P11=3.5359032967846638, P12=0.4090668334381137, P13=-0.1489007736551882, P14=-2.230562438252828, P15=2.0311935547021953, P16=0.9698005912093799, P17=0.788214876774672, P18=1.498600321808301, P19=1.8812028252495896, P20=-3.5487637939668217, P21=1.7481767477867272, P22=-1.0373553497363976, P23=-0.07659594430701136, P24=-2.1385971701471096, P25=-0.5343169638608447, P26=-2.6064223726653357, P27=3.492322445969961, P28=0.2946234479632244, P29=4.80509878303129, P30=4.330522390595576, P31=8.970831494925743, P32=3.669034747343201, P33=4.500122213397745, P34=-0.7414182657117981, P35=-1.1143904842903911, P36=0.698756276346986, P37=2.3690624709673913, P38=2.00272527492007, P39=2.5812386109773025, P40=0.7474374512549233, P41=0.782206897477487, P42=0.7074292383208587, P43=0.6862576012515758, P44=0.7072715542840708, P45=0.7530942214576053, P46=0.7217087357113553, P47=0.6958074474359166, P48=0.7001210294849526, P49=0.6997354019871903, P50=0.6793496767010093, P51=0.6985573230924312, P52=3.6814706161966606e-05, P53=1.0835283159335743e-05, P54=1.1739792737792154e-05, P55=7.733115851471363e-05, P56=0.0003393135735042618, P57=0.0006879963759204167, P58=0.0007051409979511416, P59=0.003662673217422362, P60=0.005098736692583492, P61=0.001023386980620656, P62=0.0004493266467560919, P63=0.0004470237208857061, P64=0.0001494655297335518, P65=0.00035254431883596066, P66=0.0008636830353925436, P67=0.0015963002280588437, P68=0.003744581475331332, P69=0.0028756210755966167, P70=0.0007471911390023225, P71=0.00021808918548195017, P72=0.00025205638291931747, P73=0.0005585978528825744, P74=0.0006724245166088927, P75=0.0006577851608993534, P76=0.0012517541092532639, P77=0.0022430051737997907, P78=0.0020476871266364373, P79=0.0012192849119196149, P80=0.0005829516645682086, P81=0.000619384734081707, P82=0.0008849683649011152, P83=0.0013904496996586836, P84=0.0036454559503387987, P85=0.0048881363889855, P86=0.0014748624605054861, P87=0.00032669182485186633, P88=0.00015967820458200895, P89=0.00023193243151790942, P90=0.00025957689653283264, P91=0.001274005005251008, P92=0.004155926954111847, P93=0.001037354323349427, P94=0.00020763940212302408, P95=0.0003923166826307424, P96=0.00018069835431611773, P97=0.0007147567796429236, P98=0.0010696609726761977, P99=0.00027905231089009525, P100=8.943848553113093e-05, P101=6.278814041063697e-05, P102=0.00034530621716564807, P103=0.00018812444666544429, P104=1.4646124718277228e-05, P105=2.485242655412584e-05, P106=0.0002852093860313817, P107=0.0002756131500671414, P108=1.8901198932860874e-05, P109=2.890009555157998e-05, P110=5.975313008566088e-05, P111=5.089062599293333e-05, P112=2.8470563738170654e-05, P113=2.198622765960227e-05, P114=1.536931399415824e-05, P115=1.3268443267114394e-05, P116=1.933201927599437e-05, P117=2.293197929729539e-05, P118=4.198274750973198e-05, P119=1.7333943037992956e-05, P120=1.4983168294970711e-05, P121=2.3029446164761453e-05, P122=1.8748946692524964e-05, P123=2.542071137218341e-05, P124=4.787057892792614e-05, P125=1.6044112857500012e-05, P126=6.657440342165156e-06, P127=6.342007406901681e-06, P128=3.6250281496528934e-06, P129=4.0399296917728704e-06, P130=2.4366844796927324e-06, P131=2.3371026900262236e-06, P132=1.3217094960804278e-06, P133=2.200167991655567e-06, P134=5.665739623839577e-06, P135=4.526609067304727e-06, P136=4.9593599657727505e-06, P137=5.021073804044622e-06, P138=2.097658791335882e-06, P139=1.8280275486119653e-06, P140=1.5830161179124114e-06, P141=1.5597226688054885e-06, P142=2.148268801833612e-06, P143=2.565564846671176e-06, P144=6.542244147143678e-06, P145=4.699044481589492e-06, P146=7.066907994129944e-06, P147=8.57355061830426e-06, P148=1.8699717047408434e-05, P149=8.388078403172816e-06, P150=1.2576495313007624e-05, P151=1.5622458347886945e-05, P152=4.2374696002918735e-05, P153=7.777465238115584e-05, P154=4.072212185696781e-05, P155=2.3803245689363415e-05, P156=4.210118063232564e-05, P157=3.486318630073677e-05, P158=3.704524817538713e-05, P159=2.0721443442998345e-05, P160=8.879720288436539e-06, P161=4.556626362116293e-06, P162=1.995817536818509e-06, P163=2.4054432643038922e-06, P164=6.85782715144005e-06, P165=5.496472651219421e-06, P166=3.273621715392969e-06, P167=2.166359009846936e-06, P168=1.1402578626315397e-06, P169=7.450823086234461e-07, P170=6.537525594789093e-07, P171=7.29538681804454e-07, P172=1.1360679127667328e-06, P173=1.3127458483646488e-06, P174=1.113399195978264e-06, P175=6.768049803290587e-07, P176=5.100378924754121e-07, P177=3.1761206167651363e-07, P178=1.3373198439307826e-07, P179=1.4516400076030347e-08),Row( P0=-802.3880755070053, P1=71.46448158830435, P2=12.959265393061601, P3=13.713652462421306, P4=-0.7795209078977804, P5=5.800147273395326, P6=7.406704039224814, P7=8.879269387523076, P8=3.857748571364557, P9=-1.883392912302655, P10=0.5407777475845787, P11=4.004839933906757, P12=2.3642150467588303, P13=0.10139656632351458, P14=-1.0245662170787941, P15=3.1895954607274595, P16=1.0342955519410737, P17=2.0502959261913682, P18=0.8379805956392176, P19=0.8895462251326972, P20=-2.3874176700068124, P21=1.1415308912076738, P22=-0.7449851413446369, P23=0.7835009741062513, P24=0.5603133845932149, P25=2.9204466075038895, P26=-0.5073438910453867, P27=6.097501418749142, P28=1.3278318149839927, P29=2.3791621517593904, P30=1.5250254935544718, P31=2.4309733520992003, P32=-0.45188105968375986, P33=3.1334885260991348, P34=0.7415556377809468, P35=-0.5953414997094102, P36=1.487206398413804, P37=1.3319488462063962, P38=1.8011597179277337, P39=2.258387635069064, P40=0.7358421136122748, P41=0.737840031153286, P42=0.7494749599418157, P43=0.7735606178510392, P44=0.7819696906809589, P45=0.7838731757231806, P46=0.7880880959796452, P47=0.7804864302949248, P48=0.7576746776002972, P49=0.7764061210843782, P50=0.7824453991570697, P51=0.7455859317344704, P52=5.760183654372816e-05, P53=1.4072232617697428e-05, P54=8.990117108325674e-06, P55=2.0494829345836866e-05, P56=8.850916501557938e-05, P57=0.0006100496034489911, P58=0.0006786555316337184, P59=0.000650230016936089, P60=0.002111545196612453, P61=0.0014348629431267389, P62=0.0002728981338828219, P63=0.00016080754135784034, P64=9.76524913833156e-05, P65=0.00010239141030281043, P66=9.960790784619234e-05, P67=0.00016140008310347288, P68=0.00031639204390168196, P69=0.0019772955025000585, P70=0.003896167582612135, P71=0.0013328089971123519, P72=6.745338576609681e-05, P73=4.483689120280167e-05, P74=4.2276543289721906e-05, P75=4.6052825268230014e-05, P76=3.983206582139893e-05, P77=0.00020291860037093784, P78=0.0014682832536899349, P79=0.004297953685325875, P80=0.002625948324745067, P81=0.0002970680033671587, P82=6.562239796882765e-05, P83=5.789329741943392e-05, P84=9.030200220743132e-05, P85=5.329373481390771e-05, P86=2.536419402433917e-05, P87=2.4687476384799375e-05, P88=4.952791209503824e-05, P89=5.097319766166422e-05, P90=6.369770434514452e-05, P91=5.4740562545097616e-05, P92=2.214541900770431e-05, P93=1.3150778992877012e-05, P94=2.1162929809979375e-05, P95=2.562614199538258e-05, P96=4.005333646369251e-05, P97=2.165812273047708e-05, P98=1.8326421756834305e-05, P99=1.760198548703108e-05, P100=2.6472084715162453e-05, P101=4.651840165174184e-05, P102=7.767305091134563e-06, P103=3.484410152393974e-06, P104=7.097019429987898e-06, P105=9.79143787634663e-06, P106=6.141746203632738e-06, P107=4.695888425605864e-06, P108=7.425805330633825e-06, P109=6.4905881128165095e-06, P110=3.6389964835156862e-06, P111=9.263238336702175e-06, P112=4.238217133965219e-06, P113=1.501332863241769e-06, P114=1.8431627917019582e-06, P115=1.7759149786301132e-06, P116=2.4145555057706695e-06, P117=5.652287240456561e-06, P118=3.729916875178349e-06, P119=1.7709025930794954e-06, P120=2.3207183523338912e-06, P121=4.8035152505403925e-06, P122=2.9966968752820343e-06, P123=2.2163218606105803e-06, P124=2.582421491442807e-06, P125=2.114856872946529e-06, P126=4.751143218888189e-06, P127=1.008774351391838e-05, P128=3.2037315043717076e-06, P129=3.73671148397014e-06, P130=2.6895974301912578e-06, P131=2.416857046178744e-06, P132=7.917624699729693e-07, P133=1.1527113763522613e-06, P134=2.255035274958872e-06, P135=1.746802590042826e-06, P136=4.104551138724787e-06, P137=2.643691926446612e-06, P138=2.7667784590090787e-06, P139=2.195172934361227e-06, P140=7.975925593009802e-07, P141=1.1924909277927204e-06, P142=7.931940911815174e-07, P143=7.12422049026984e-07, P144=2.2598344925115123e-06, P145=1.595476687068505e-06, P146=1.4962821957572835e-06, P147=1.0175355077840748e-06, P148=1.3255287710304704e-06, P149=1.3564349841353386e-06, P150=1.6904356966956589e-06, P151=1.6231142731005557e-06, P152=1.7361293819434013e-06, P153=2.47573558230752e-06, P154=2.251111179696501e-06, P155=3.3024209068338003e-06, P156=6.962381447770715e-06, P157=1.0009340311115204e-05, P158=2.4035611739250625e-05, P159=2.1159709273273938e-05, P160=9.499009682546143e-06, P161=4.476511830301374e-06, P162=2.7053012022426996e-06, P163=1.0520055840546474e-06, P164=3.0068917291207983e-06, P165=3.6424649597002785e-06, P166=2.9472005305734467e-06, P167=1.3324755648790095e-06, P168=5.24370715691946e-07, P169=3.353435015514486e-07, P170=2.786331435118763e-07, P171=3.8854128492139943e-07, P172=5.299742127286869e-07, P173=7.143783948854263e-07, P174=6.607556332451777e-07, P175=3.52714724520863e-07, P176=2.2063552306909868e-07, P177=1.4063187929521623e-07, P178=5.397925661084176e-08, P179=5.381185466314545e-09),Row( P0=-773.6579256980472, P1=74.247412862561, P2=11.48311334107323, P3=9.99770333208192, P4=-6.684171412561015, P5=5.790944872095046, P6=6.260989976722059, P7=4.098460547735149, P8=1.6501540878638286, P9=-3.088804226945599, P10=-2.1554021014645763, P11=4.553871716423664, P12=-0.7364746025156601, P13=-0.08292022442913217, P14=-4.406544896484506, P15=2.341156686556778, P16=2.124630210035994, P17=-2.554773500661894, P18=1.0203211620857437, P19=1.0351566540616373, P20=-4.225743394800558, P21=3.1315749897401517, P22=-3.034583098200547, P23=0.5774772771681228, P24=-1.2070167600515216, P25=-0.04946189787012228, P26=0.775342597784681, P27=5.292251972431363, P28=3.0822245774051416, P29=8.26193519844005, P30=4.092270014456283, P31=7.555202334669374, P32=1.9733297748869063, P33=3.7047155261623197, P34=-2.7229274285588887, P35=-1.7696901866874748, P36=-0.12557631897609955, P37=2.905735015095525, P38=0.5235249839412481, P39=1.9143690899970411, P40=0.7293749390109581, P41=0.7863545305729722, P42=0.7179638982588765, P43=0.6519800558552187, P44=0.6706984383979749, P45=0.681566153824337, P46=0.6750305344667733, P47=0.6559344143026715, P48=0.6926498124693438, P49=0.6575185041685747, P50=0.6145627538800928, P51=0.6732711766093569, P52=4.792971045683222e-05, P53=9.438723162829624e-06, P54=1.7278849231088808e-05, P55=6.060665799014827e-05, P56=9.47774245973679e-05, P57=0.00017783152766233574, P58=0.0004209195492413276, P59=0.0034598770721971815, P60=0.010633022640988561, P61=0.003357735779340904, P62=0.00024805814621362357, P63=0.0002706120906383498, P64=0.00016758621281977957, P65=0.00020226668988190257, P66=0.0006384942234187534, P67=0.001616256433431177, P68=0.002358023026315174, P69=0.004606965533223835, P70=0.002374762773003942, P71=0.0002449750549859996, P72=7.217968046598749e-05, P73=8.233151041757982e-05, P74=0.00020225117631370972, P75=0.0004524918149171169, P76=0.0004203941322609721, P77=0.0009439667407976377, P78=0.0024059583419664783, P79=0.001888183332448418, P80=0.0003414950700625413, P81=0.00021081159785576393, P82=0.0005456596771463864, P83=0.0008786194000524012, P84=0.0007067886253100882, P85=0.0007069576151182778, P86=0.0005877757833759904, P87=0.0007798731734700894, P88=0.00025598030024498556, P89=9.976181805716023e-05, P90=0.0004423289639243133, P91=0.0004173186733624773, P92=0.00026041004266639183, P93=0.0004546272878443638, P94=0.0008253667800186571, P95=0.00017695020017703068, P96=0.00019173428700503757, P97=0.00014372585749540913, P98=0.00023616223053893594, P99=0.0005667105305749692, P100=0.0003775738488089781, P101=6.388123483337271e-05, P102=3.058234547771444e-05, P103=8.576939840851887e-05, P104=0.00016140075424292527, P105=3.7503162351909894e-05, P106=1.7294429290057418e-05, P107=4.379287867155246e-05, P108=7.57599381359163e-05, P109=2.6454466502645234e-05, P110=5.334736233994247e-06, P111=2.209302080845389e-05, P112=1.5197636971091678e-05, P113=5.4972951208222775e-06, P114=7.81189252947323e-06, P115=6.014975177115489e-06, P116=9.009351291446742e-06, P117=1.4632758384112335e-05, P118=6.3958806900374915e-06, P119=6.284568520244994e-06, P120=8.884086069309122e-06, P121=1.7116051024614018e-05, P122=1.1431365501981736e-05, P123=8.461830833852282e-06, P124=7.803154780262266e-06, P125=3.976569193319223e-06, P126=2.411074524490563e-06, P127=1.8340831368281513e-06, P128=2.0046930750509906e-06, P129=1.5382753291088733e-06, P130=1.4884459389073792e-06, P131=2.0719827293414485e-06, P132=1.2850448905185042e-06, P133=2.1025308337488603e-06, P134=2.858345365383624e-06, P135=2.926609805404698e-06, P136=6.5328904183269305e-06, P137=4.636802543415889e-06, P138=5.717792075460962e-06, P139=3.3364958882010167e-06, P140=1.7238837929931593e-06, P141=1.964394838075584e-06, P142=1.8828481973445017e-06, P143=2.243310726321491e-06, P144=5.149821554768783e-06, P145=7.189900598579786e-06, P146=6.8276535255321295e-06, P147=6.407669327381953e-06, P148=9.708283304817203e-06, P149=1.4313089361209775e-05, P150=9.88865643117857e-06, P151=9.592577958866525e-06, P152=9.315178315973813e-06, P153=9.289371863057808e-06, P154=9.238802410419297e-06, P155=1.2229000733558809e-05, P156=1.413100035575131e-05, P157=2.2696957489504015e-05, P158=2.842318954687765e-05, P159=2.6045320837089073e-05, P160=2.8404618461590684e-05, P161=1.621678837323871e-05, P162=8.38482142748139e-06, P163=3.2347151199136974e-06, P164=3.214115923776534e-06, P165=2.691937957838633e-06, P166=3.2662992081468557e-06, P167=3.2786255858773586e-06, P168=3.010068095395114e-06, P169=1.6585093639553679e-06, P170=1.3180973805169797e-06, P171=1.1668250822346545e-06, P172=1.2385161371606884e-06, P173=1.701325866883051e-06, P174=1.4094626322649506e-06, P175=1.3364783682195176e-06, P176=1.0387375992570792e-06, P177=8.191006915186666e-07, P178=2.447479772044309e-07, P179=1.6664405057957448e-08),Row( P0=-579.5724663771329, P1=56.575805059481446, P2=-15.236847255068433, P3=-1.9813903521549823, P4=-13.949616768390424, P5=-15.448130596340324, P6=-8.859491751137906, P7=-1.8516654030597766, P8=-1.7677835959864676, P9=-0.26733153240337854, P10=-12.970990392990743, P11=-4.594309516002083, P12=0.13801047516225226, P13=-7.47564488909213, P14=1.038561410697735, P15=6.301802598948296, P16=3.5266720786207375, P17=8.593284435920168, P18=5.139873706923341, P19=7.850744603775326, P20=3.426140233643139, P21=3.3423173415553915, P22=5.832820599561897, P23=5.67836061425704, P24=2.8301984829111735, P25=6.476408584696751, P26=1.2089273975414265, P27=4.4400265589270855, P28=2.1093290874731174, P29=0.5238713960161955, P30=-0.5348508042695433, P31=4.157848989143234, P32=0.563282657893698, P33=4.692622951193732, P34=-1.4751442796612126, P35=0.26875016375306376, P36=-2.3091528940749324, P37=2.2163665784401383, P38=4.159681692616503, P39=2.000163198170927, P40=0.6533131052818913, P41=0.6360031418585196, P42=0.5622325181745692, P43=0.5843438096836169, P44=0.6476110301226453, P45=0.6814955063589587, P46=0.6618014534158715, P47=0.6630297609218504, P48=0.6545194114313463, P49=0.67695958855671, P50=0.6562226614237202, P51=0.6253052786782797, P52=0.0012939741173917032, P53=0.00014167685970257058, P54=4.230962225000116e-05, P55=8.488972686143928e-05, P56=0.00012273417324762037, P57=0.0001618279443305467, P58=0.00019863759181727066, P59=0.0006750676528759182, P60=0.0034812562036917317, P61=0.010255274516664862, P62=0.023973080195828753, P63=0.021890909686419668, P64=0.02367646625879895, P65=0.05782602427583747, P66=0.06081288548062864, P67=0.07164616720240248, P68=0.09144804774654174, P69=0.055582655855157675, P70=0.01569030387706017, P71=0.008717372345510992, P72=0.017316457708481784, P73=0.03962510127866809, P74=0.028939935230716512, P75=0.007467768473146861, P76=0.012797862772201091, P77=0.06768373300370245, P78=0.24822556340550525, P79=0.3771202684540445, P80=0.19116896637826286, P81=0.1008767707587476, P82=0.10933905047176481, P83=0.14768003414968442, P84=0.28143983625572017, P85=0.22455437806540962, P86=0.10209802946783587, P87=0.03954917265378715, P88=0.027018909187829267, P89=0.019245865759613144, P90=0.03481010909536451, P91=0.14127208316698392, P92=0.24526322858535557, P93=0.0655503933521231, P94=0.031990444974066994, P95=0.030375677397530075, P96=0.024172352959390835, P97=0.029075014652440848, P98=0.020185751486212943, P99=0.013969596863765595, P100=0.017139070513686166, P101=0.010699849791126786, P102=0.004101641722168062, P103=0.007928150591753943, P104=0.004981371394431743, P105=0.005982800602854307, P106=0.017424349373971602, P107=0.0132853877063196, P108=0.0047157642330077215, P109=0.004348208502135887, P110=0.00990692502661528, P111=0.013495314025934072, P112=0.011757136859413403, P113=0.014662736913818279, P114=0.004809229880542902, P115=0.008534160069906779, P116=0.005115264473869339, P117=0.00977644386143837, P118=0.013211210771957829, P119=0.0076504682942836345, P120=0.0035708847656679365, P121=0.006431200973559303, P122=0.002533313396176448, P123=0.0023456840754700193, P124=0.01122598290324931, P125=0.004961254763006617, P126=0.001206007917450266, P127=0.0008707861267995185, P128=0.0007996239238455183, P129=0.0011412012753693245, P130=0.00021464009239888676, P131=0.0006262386898897549, P132=0.0030701053408235673, P133=0.0016017859016161547, P134=0.002971793044985338, P135=0.004165053912609409, P136=0.0013999160872829835, P137=0.0009974611234335271, P138=0.0010548925274533744, P139=0.0006871494691396034, P140=0.0007350785383143768, P141=0.0008345457644791199, P142=0.0006696075204168145, P143=0.0009583977430305685, P144=0.0005348407277922274, P145=0.0006733407693623522, P146=0.00024089833782461398, P147=5.3426376045885464e-05, P148=4.0104464641407416e-05, P149=3.2201751179987366e-05, P150=5.6371122559543094e-05, P151=0.00014287690613188538, P152=0.0006552851855531722, P153=0.0002634171154045154, P154=0.000669186909661973, P155=0.001210023425414539, P156=0.0005268723931404613, P157=0.00039733526233427753, P158=0.0004738725372599511, P159=0.00102029643144126, P160=0.0011486632726675392, P161=0.001360327514181489, P162=0.001000319861523751, P163=0.0006561932563489986, P164=0.0007971908389108978, P165=0.0004977885860397779, P166=0.0004523754958287067, P167=0.0003673548626327031, P168=0.00026077717240984146, P169=0.00017078388521233655, P170=0.00010389188250218649, P171=6.12407029402775e-05, P172=9.366361668312579e-05, P173=0.00014862177492334117, P174=0.00012642518397625445, P175=0.00013357064710437834, P176=0.00011901726537932162, P177=6.542548468268183e-05, P178=2.2985417690708983e-05, P179=2.0380942361855462e-06),Row( P0=-789.4511934019533, P1=79.58302408385774, P2=10.023392610190328, P3=15.155368185735906, P4=-4.08300283983331, P5=6.527002822558591, P6=5.965744442376787, P7=5.4147448244224, P8=3.5223770064829774, P9=-0.9629648917768249, P10=1.890327944120607, P11=1.363105064283673, P12=0.6608083415611633, P13=1.2303087274859914, P14=-3.946839664317315, P15=3.527476473803214, P16=0.8387147219531648, P17=-0.28297342303959466, P18=-0.5745491665221995, P19=2.0909754697288827, P20=-4.716704804641007, P21=1.7748447116710753, P22=-1.1453443567295563, P23=2.496245963487494, P24=-1.8076020681571452, P25=0.36856911636152745, P26=-2.272808750187624, P27=2.543928745267149, P28=-2.8583803714400577, P29=-0.9286890668809993, P30=-2.043962296667473, P31=3.035884456934731, P32=0.21581439597916827, P33=5.805977594728847, P34=3.512271148058161, P35=3.984825911098915, P36=3.7381721355071362, P37=4.939786291639042, P38=2.7412025677909013, P39=4.291428041772433, P40=0.7444853184715514, P41=0.7228557536018181, P42=0.6883836841141928, P43=0.6820372141334098, P44=0.7070539400455851, P45=0.7358471664780935, P46=0.7433324125081667, P47=0.7258653547593547, P48=0.7182186397035392, P49=0.7646784808113125, P50=0.7665948856725132, P51=0.7812392676828899, P52=4.331919922919097e-05, P53=9.47229005325626e-06, P54=8.708663988501546e-06, P55=1.0339289313201262e-05, P56=6.584140703414472e-05, P57=0.0007224028028507995, P58=0.002285616546794587, P59=0.003009092513619911, P60=0.0008523379453094722, P61=0.00016910156155331056, P62=0.00013680980725686866, P63=0.00014802918240513452, P64=0.00030763321382952296, P65=0.0009079810158481791, P66=0.0021732153168640134, P67=0.002856118462613435, P68=0.00134716766161022, P69=0.00022831826834108148, P70=0.0001479505151277181, P71=0.00017511959125496467, P72=0.00022026423921212585, P73=0.00031265747563320414, P74=0.0006269673787841038, P75=0.0012525931531096783, P76=0.001158935863320818, P77=0.0003646350740869535, P78=0.000327481302436241, P79=0.0003840704706118497, P80=0.0001833312228612966, P81=0.00021465486928693025, P82=0.0004362519041277261, P83=0.00147798525722541, P84=0.0018742627369887463, P85=0.0006357646203284624, P86=0.00017572401080504117, P87=7.332191865884439e-05, P88=0.0002083544787274983, P89=0.00013827659527673253, P90=0.00014771785595596752, P91=0.00021598356163650992, P92=0.00028052198279740216, P93=0.0002145249423745442, P94=0.00029568272122471864, P95=0.00012082429054808664, P96=6.824590484590763e-05, P97=6.447466653681003e-05, P98=0.0002459726778192085, P99=0.0004371255702943099, P100=9.693047249791244e-05, P101=4.1670658082622256e-05, P102=6.841899937460212e-05, P103=0.00012741237881792636, P104=2.1508372179806852e-05, P105=1.1475491884253195e-05, P106=0.00014002304918754916, P107=0.00013972295947042163, P108=2.0796489704418843e-05, P109=1.720863387543819e-05, P110=1.3906879461061556e-05, P111=1.5166533211333917e-05, P112=1.0021150830641807e-05, P113=5.2140624891561916e-06, P114=4.281894912457945e-06, P115=3.6002487755216557e-06, P116=7.532333865776394e-06, P117=1.7157989299682182e-05, P118=9.759606113870227e-06, P119=5.2600425002277714e-06, P120=4.3255251535596715e-06, P121=7.081171526361814e-06, P122=8.470227033424731e-06, P123=4.041874228981025e-06, P124=1.0334670988056114e-05, P125=5.6034044634994e-06, P126=2.153345421382271e-05, P127=7.722955485501819e-06, P128=1.2768248164305339e-05, P129=7.791532348492782e-06, P130=7.899427078430565e-06, P131=5.551449352447308e-06, P132=4.059955826036351e-06, P133=7.933696201777079e-06, P134=1.4226607759407795e-05, P135=9.764385922275135e-06, P136=8.69385496805649e-06, P137=5.766085095966685e-06, P138=2.0169938427224384e-06, P139=2.162486651774333e-06, P140=1.1690960607511874e-06, P141=1.01531020915614e-06, P142=1.0814646943172714e-06, P143=1.2194324194763323e-06, P144=4.779001837177302e-06, P145=3.525245835209206e-06, P146=2.2730369377213686e-06, P147=2.2720096074884892e-06, P148=5.303565575865411e-06, P149=1.056692678503491e-05, P150=8.419555275819486e-06, P151=4.6073534432556675e-06, P152=5.364896279656821e-06, P153=6.965299945383282e-06, P154=4.427892856464376e-06, P155=5.152033191760864e-06, P156=8.813996442361178e-06, P157=1.3140683660033636e-05, P158=1.3791166466025704e-05, P159=1.2556492054833493e-05, P160=7.5141221226078785e-06, P161=4.14264669449995e-06, P162=1.5303124395943271e-06, P163=6.374166424183902e-07, P164=3.6126156939541533e-07, P165=4.2987670949005653e-07, P166=4.3100610057946083e-07, P167=4.005297868655459e-07, P168=2.1624222790943028e-07, P169=2.0936468088654608e-07, P170=1.6948908600317421e-07, P171=1.4051089676475555e-07, P172=3.9580288358186956e-07, P173=5.535549126058224e-07, P174=4.231333770348371e-07, P175=2.6943583190885775e-07, P176=2.5191626320464696e-07, P177=1.8321594337501534e-07, P178=5.079350618907265e-08, P179=4.327299987053656e-09),Row( P0=-584.1397370145841, P1=42.49040711209298, P2=-10.256689682497598, P3=-2.854690029995877, P4=-10.960356629723679, P5=-10.547138480822115, P6=-8.826355300905208, P7=-2.717010027016366, P8=-3.1468869645063573, P9=-0.7790508662656702, P10=-7.819236399501585, P11=-7.6241393145686445, P12=1.0218131327497502, P13=-3.6562081629852496, P14=4.28986468677627, P15=11.27760465559502, P16=9.269206097549885, P17=11.63431276239393, P18=7.4840671539384855, P19=5.798798169571055, P20=0.35377074647964507, P21=-0.3083677054569247, P22=2.9335713867043833, P23=4.148934474390523, P24=-1.1758083011267417, P25=3.0947604668120667, P26=-2.3848204402581024, P27=1.3095401440126277, P28=1.0058796698602066, P29=1.9673590095358668, P30=1.570377200179336, P31=4.244934178600041, P32=0.1689063872204491, P33=3.3614551936694745, P34=0.48181284836497135, P35=3.106039183523822, P36=0.1866625163846534, P37=1.9746708170123763, P38=1.4319822524127732, P39=1.7857053278777357, P40=0.7289608708410822, P41=0.662946608085295, P42=0.653904224272271, P43=0.6412847688722857, P44=0.6392336533135602, P45=0.637997266205935, P46=0.6223641173953577, P47=0.6373953548716275, P48=0.6269824964324954, P49=0.6518943852794856, P50=0.6879833748627957, P51=0.7270034533787407, P52=0.0005498037816520923, P53=0.00014090919768288942, P54=4.694937137745426e-05, P55=8.907677330943908e-05, P56=0.00015294441456072182, P57=0.0006189081559103651, P58=0.0006117789450845372, P59=0.00040209929070483054, P60=0.0008426586300886686, P61=0.0023229277452813273, P62=0.010394157442752348, P63=0.02298999003312527, P64=0.023736305585949545, P65=0.05028076962888237, P66=0.11044492627297199, P67=0.1512854519304076, P68=0.11174508875557301, P69=0.08112814100919011, P70=0.05182768815887242, P71=0.014322991779731016, P72=0.0037463081953373442, P73=0.00327798756456757, P74=0.004720667869156185, P75=0.008047500303226278, P76=0.012687687263638421, P77=0.033624979176137125, P78=0.06517336647975365, P79=0.09932150074339034, P80=0.11227028709919175, P81=0.20778436060345212, P82=0.20224112652178855, P83=0.1355922156729394, P84=0.38798764849089706, P85=0.5131519069119946, P86=0.2136883934837885, P87=0.15080629069729892, P88=0.06321271154159556, P89=0.013521700910918153, P90=0.005720634838382801, P91=0.0063879619178185765, P92=0.032145970488159314, P93=0.05058137938859644, P94=0.044632415392907104, P95=0.023489881277706536, P96=0.03982286496606598, P97=0.04277178791042421, P98=0.01830570608369354, P99=0.01925190870753846, P100=0.01590988855593872, P101=0.004479228457668505, P102=0.007823729782067353, P103=0.011600971189124972, P104=0.004076757874113783, P105=0.013529626977379295, P106=0.01796594311911068, P107=0.017572623599200005, P108=0.020906683647563512, P109=0.034879175961097054, P110=0.012658104503967878, P111=0.005755026799334616, P112=0.010741233164696076, P113=0.002464933219721152, P114=0.002531500190333747, P115=0.004100554847439126, P116=0.0034980161326958804, P117=0.014733229358811797, P118=0.01226494135958437, P119=0.005573774410323518, P120=0.002072209937463407, P121=0.0016301431044839242, P122=0.0034643949953729555, P123=0.0024442010105033447, P124=0.002872537665380728, P125=0.0021756038993174564, P126=0.0007325375381748744, P127=0.0004975230085521829, P128=0.0009138688716286595, P129=0.0006796848387127883, P130=0.0006828614604131484, P131=0.0006977784780394012, P132=0.0014169001038806443, P133=0.0009881332538234674, P134=0.0018519495859650095, P135=0.0014395009828037679, P136=0.0007584041197571689, P137=0.0004988238938416347, P138=0.00044060676233107643, P139=0.0002716119769317927, P140=0.00032052939022004216, P141=0.000289829537816601, P142=0.0004399468905643314, P143=0.00032795481677824057, P144=0.00023535260432096277, P145=0.00011314028393345672, P146=7.79487511278792e-05, P147=3.556692737114347e-05, P148=4.502311519469006e-05, P149=7.177409942289488e-05, P150=0.0001141984512401417, P151=0.00015627238940402416, P152=0.00028544929948103287, P153=0.0007097366192677668, P154=0.0008878986862652112, P155=0.0010902771295745906, P156=0.001028404790114393, P157=0.0011055902846403422, P158=0.0011274144815495437, P159=0.001109839685231641, P160=0.0011016213741644252, P161=0.0012540152235226492, P162=0.0009133060851250328, P163=0.0006592206172913443, P164=0.0006372626444143614, P165=0.0005460717659577101, P166=0.0003264984655720959, P167=0.0002571831158785381, P168=0.00022359321737103855, P169=0.0001372714814299263, P170=8.295843274476113e-05, P171=0.00010358034955056827, P172=0.00018175308162454324, P173=0.00022883141258734656, P174=0.0002892677627799928, P175=0.00023997968696917076, P176=0.00022580851854215654, P177=0.00015002137496917167, P178=5.310658338895397e-05, P179=5.45293792285839e-06),Row( P0=-743.9140582201055, P1=62.17795923300835, P2=6.012755536832761, P3=9.789493672621148, P4=-4.50024593917879, P5=3.554486335614906, P6=8.422895099084204, P7=6.166286134698221, P8=5.569513208250847, P9=1.3233845651246317, P10=0.12408060909999943, P11=1.3253138442205272, P12=2.603299822781516, P13=-1.4349467762786385, P14=-0.494679594996693, P15=4.150105449683692, P16=2.2002852107351916, P17=1.7633136065847692, P18=2.079611084510063, P19=2.8025379935728654, P20=-1.804029572152963, P21=1.6881857237639428, P22=-0.750064110876157, P23=0.9263455726360686, P24=-1.0421080663690843, P25=0.8011930172694381, P26=-0.8551787893988259, P27=2.6466140699873604, P28=-0.955729391103623, P29=1.702340606817791, P30=-0.28997510777021873, P31=4.715302541417443, P32=-0.7522607544839666, P33=2.170509146174462, P34=-1.5201611861709645, P35=-0.7087867834170818, P36=-0.7375711751726787, P37=-0.20134427889427192, P38=0.6434400509120033, P39=1.140364155528779, P40=0.7284650316608196, P41=0.739081897462876, P42=0.7124178315019486, P43=0.7139874481515674, P44=0.7480898671336791, P45=0.7745756325272007, P46=0.7606870739896957, P47=0.7798846113123445, P48=0.7510872593693346, P49=0.7328846304162459, P50=0.7313344904700904, P51=0.713446277138323, P52=0.00013649658738554814, P53=2.079666434087277e-05, P54=8.955478614997842e-06, P55=4.174296974302157e-05, P56=0.00038336886164907504, P57=0.0012095602927166542, P58=0.0005656299257423418, P59=0.0017389779350227995, P60=0.0034790978166982837, P61=0.00097859415277236, P62=0.0004895556248088623, P63=0.000552108332188679, P64=0.00018526027622545513, P65=0.00012902098892860563, P66=0.00046832211996182055, P67=0.001574753328868553, P68=0.0067278793259201285, P69=0.0061812535942017514, P70=0.0010332675230625213, P71=0.00013365199822021578, P72=6.520528900375652e-05, P73=9.994062621923452e-05, P74=0.0001406098412970746, P75=0.00029997444671022095, P76=0.0015681461899600272, P77=0.008274521308883549, P78=0.006908968760640031, P79=0.0018821705308676247, P80=0.0003645013207128858, P81=0.00022308684137836596, P82=0.00026897544743494756, P83=0.00031342256503532874, P84=0.00045376453243386013, P85=0.0005982409733969394, P86=0.0004969540297753834, P87=0.00016665492479319942, P88=8.957064235248854e-05, P89=7.801498982247185e-05, P90=0.0002864109061963258, P91=0.0005887325589682639, P92=0.0004002566228987747, P93=0.00017654637513731157, P94=9.414760629537673e-05, P95=0.00016223367202011755, P96=0.000709726728049139, P97=0.0005905965527060685, P98=0.00010914582172743699, P99=9.45370904560187e-05, P100=0.00019433760964184216, P101=0.0007078382026127625, P102=0.0003770139642476182, P103=6.162273600178891e-05, P104=6.507647119580684e-05, P105=0.0004121145246062579, P106=0.0004242008238218292, P107=6.756501841918113e-05, P108=5.607285376501293e-05, P109=4.030304085181087e-05, P110=8.479094699069609e-06, P111=3.0396677341292712e-05, P112=1.7276028323282083e-05, P113=1.0249622270205356e-05, P114=5.893541129116412e-06, P115=7.9621053277717e-06, P116=9.350378499774124e-06, P117=1.935837851822854e-05, P118=1.900211966752757e-05, P119=1.0430937725040524e-05, P120=8.624630883761652e-06, P121=1.266992372472911e-05, P122=1.0194636927908021e-05, P123=8.334898075038922e-06, P124=1.937976713165035e-05, P125=1.107925772820473e-05, P126=3.1037271094084e-05, P127=2.5521744565143652e-05, P128=4.602642779698944e-05, P129=2.7110382257615596e-05, P130=4.3934173246425327e-05, P131=4.514888766468931e-05, P132=1.906320430664689e-05, P133=2.4648946246936114e-05, P134=4.625742229332431e-05, P135=5.253382904657858e-05, P136=6.521351180757961e-05, P137=7.550201434006762e-05, P138=8.105778186154456e-05, P139=8.972869915501632e-05, P140=8.022608540155269e-05, P141=3.195181769419763e-05, P142=2.139538434084372e-05, P143=1.8751372979379573e-05, P144=3.865201549949267e-05, P145=2.8570832828995674e-05, P146=2.1637094829104973e-05, P147=1.6227367268013404e-05, P148=1.7269994004126568e-05, P149=3.267758737030026e-05, P150=7.519629842482431e-05, P151=8.537693679597711e-05, P152=6.101575336407783e-05, P153=2.3904319461456583e-05, P154=1.653295102879227e-05, P155=4.3523155419713154e-05, P156=9.560552375496148e-05, P157=0.00012259702029304153, P158=0.00012170690726550296, P159=0.00019276896875121015, P160=0.00010767548481922051, P161=0.0001174273873594304, P162=5.63694337605275e-05, P163=1.8530872779249765e-05, P164=5.148430377780463e-05, P165=4.5341541864168165e-05, P166=3.59497046105625e-05, P167=4.303146247119446e-05, P168=3.0400393354428274e-05, P169=1.1538832451079137e-05, P170=5.77818404010939e-06, P171=6.628672625343236e-06, P172=1.0060233429535882e-05, P173=2.90767926902937e-05, P174=2.7562160500409006e-05, P175=2.5800268549219812e-05, P176=2.2199937260246057e-05, P177=1.0462859774981872e-05, P178=5.99980267062384e-06, P179=4.849276774036348e-07),Row( P0=-498.80805824425187, P1=24.753498844069156, P2=-26.902455293111714, P3=-7.500432825581887, P4=-14.74704443107054, P5=-13.975855111431404, P6=-7.142791219291118, P7=-0.1696837844032619, P8=-3.2774902455112613, P9=0.28168884051275916, P10=2.7857679597624383, P11=5.357214457076993, P12=11.812295631685833, P13=8.461290074315068, P14=6.876305145252653, P15=9.726755515623772, P16=5.979641203280625, P17=8.915735528908954, P18=4.731411590098985, P19=7.474798197330019, P20=1.5655536100482534, P21=3.255536226601265, P22=3.0288138463232337, P23=3.445168280947828, P24=0.24382552099108232, P25=2.2231029462759397, P26=1.2999193636747177, P27=2.574073400567321, P28=1.0637945293131146, P29=3.8105976644296877, P30=0.7598213148354294, P31=5.724938901396878, P32=1.8698326173779878, P33=2.855106063064815, P34=2.4487419942946937, P35=4.254477246537751, P36=2.9598606945806183, P37=2.251122090567845, P38=2.0292460238891, P39=2.091115014773912, P40=0.6468205804361677, P41=0.6006788802220111, P42=0.5895248706775054, P43=0.6349882116770669, P44=0.6105301753922098, P45=0.6134568090618682, P46=0.6596286041906331, P47=0.64159396045222, P48=0.6585539811559993, P49=0.6525458861229703, P50=0.6471649720677188, P51=0.6488545877175511, P52=0.01833389744746928, P53=0.0028473410507566348, P54=7.65274991973834e-05, P55=6.50280513520014e-05, P56=6.235772951631965e-05, P57=7.624458416161113e-05, P58=3.743407780887161e-05, P59=7.294153051539727e-05, P60=0.00011257980653977282, P61=0.00013047689209084206, P62=0.00022823537019331106, P63=0.0003839523796342512, P64=0.0007332307717196996, P65=0.010204924405823986, P66=0.06067787247933174, P67=0.11399821102278049, P68=0.0817065896538864, P69=0.11551078294650609, P70=0.21492069752278659, P71=0.2673461760909496, P72=0.2759369631447216, P73=0.4168609792435778, P74=0.5185905027068194, P75=0.5271678468859182, P76=0.8486513996977156, P77=0.6605256572699604, P78=0.46624790622422113, P79=0.4322528905503746, P80=0.18565240483116666, P81=0.10568992448156908, P82=0.17859566518762837, P83=0.23259114325573083, P84=0.4679370786149451, P85=0.7062409662249565, P86=0.6324983625215206, P87=0.9391108101504317, P88=1.6915251914608163, P89=1.0349116745435403, P90=0.8401404391857378, P91=0.6991402313432458, P92=0.23112352421797314, P93=0.41580104426314296, P94=0.8414203747724995, P95=0.7260515370268255, P96=0.20394268482656414, P97=0.24562494989428071, P98=0.3151759934059524, P99=0.23750449429960177, P100=0.19633246338858334, P101=0.25418697060026174, P102=0.144035108087017, P103=0.09436911559414839, P104=0.01844344177852226, P105=0.031870834100379164, P106=0.11865997924598695, P107=0.24089364469578775, P108=0.07233503242094569, P109=0.026429203434731027, P110=0.016229501537511255, P111=0.07252310291460266, P112=0.07253604959875333, P113=0.028635429803930013, P114=0.03335989911494503, P115=0.08056947086722267, P116=0.19179354396078335, P117=0.052105430139161135, P118=0.0602198372148118, P119=0.04196278639718177, P120=0.015349059556989515, P121=0.03257138367631237, P122=0.025706841939687225, P123=0.03251753175062548, P124=0.04902070514906082, P125=0.15338903629301257, P126=0.05980061848677457, P127=0.052131754447970115, P128=0.02737726853519583, P129=0.013116200625694921, P130=0.0408441387233254, P131=0.11638489194488637, P132=0.057991594599876045, P133=0.054764778572258976, P134=0.03548084267608449, P135=0.02335103631851827, P136=0.008933300767619808, P137=0.012545134772542833, P138=0.007997939203973554, P139=0.017099745746767148, P140=0.015131459449794618, P141=0.00698467377216832, P142=0.00669863293321872, P143=0.003615748103152815, P144=0.004019871092716995, P145=0.002592710716069548, P146=0.0014432632744500358, P147=0.0015973234970202822, P148=0.0016568008780923323, P149=0.0019888355429528303, P150=0.0026147635905943625, P151=0.003385173349525052, P152=0.0027332361107171416, P153=0.002348726884368454, P154=0.0014852065979347137, P155=0.0018879089014834749, P156=0.002516467241182171, P157=0.0030077398033933516, P158=0.0035658093117511866, P159=0.0029731934723416986, P160=0.00205387415913371, P161=0.001397202347791859, P162=0.001245825199042677, P163=0.0009973711965386883, P164=0.0014261548807329677, P165=0.0014174600649074776, P166=0.001630791634202062, P167=0.001797855010156516, P168=0.0012903989400703223, P169=0.00113329072233475, P170=0.0013264607151041179, P171=0.0007960688731978757, P172=0.0009599830396851613, P173=0.0020208756936247597, P174=0.002459863188032906, P175=0.0015837015204655835, P176=0.0009534392241392607, P177=0.0008491521528304392, P178=0.0005397613004451537, P179=6.712601703112699e-05),Row( P0=-478.2775999442751, P1=32.499035757762364, P2=-34.22599894997405, P3=-6.617626098769521, P4=-21.366302303073528, P5=-16.79281094094251, P6=-9.719202838538802, P7=0.32425952754608023, P8=-4.718678667709957, P9=-5.906849881348375, P10=-4.793627988597868, P11=-1.4887211620626812, P12=9.08074956362153, P13=5.807010832626459, P14=12.3178663710174, P15=17.284973119419995, P16=11.03491778404609, P17=12.631836467943394, P18=1.902523302095307, P19=5.512909178442214, P20=2.456389768290808, P21=3.5745136587296638, P22=7.1079115944851825, P23=4.443207647291582, P24=-1.035168175081442, P25=0.14295665663541265, P26=0.39248906743605233, P27=2.963189794159127, P28=2.0824101283566185, P29=3.861804304318541, P30=0.1703335073952723, P31=5.688415724401998, P32=0.022277310642867927, P33=3.402782527915501, P34=0.914204328165164, P35=0.505797055173206, P36=1.2326739446972852, P37=1.6713055093189442, P38=0.1116904416147457, P39=2.2302846402476737, P40=0.5716671503509363, P41=0.6608989110793843, P42=0.6838786337556534, P43=0.6458792654113309, P44=0.5795097964333354, P45=0.5989514851135999, P46=0.6001946528409606, P47=0.6127501163213573, P48=0.6422181534301885, P49=0.5968486561908543, P50=0.6035822719792413, P51=0.5996956387665541, P52=0.008444827632099038, P53=0.0010181663336671666, P54=5.126720855370871e-05, P55=4.667488623185106e-05, P56=3.93159042148195e-05, P57=4.666284152438147e-05, P58=2.6720327051600186e-05, P59=1.8116300101798223e-05, P60=3.4589571870656444e-05, P61=0.0001002128961441805, P62=0.0011868973086772755, P63=0.003766083002043415, P64=0.004528022961869519, P65=0.02902669539353967, P66=0.08749718852767517, P67=0.16089857941253982, P68=0.286806693977908, P69=0.5761864557371701, P70=1.1052948645266092, P71=0.6643335264416534, P72=0.07932084361500753, P73=0.053494659601376324, P74=0.05615937079853899, P75=0.10943893423747822, P76=0.21305140916679266, P77=0.14372085826170086, P78=0.04580686291197768, P79=0.025070772871163088, P80=0.01914743870126788, P81=0.035900679867833594, P82=0.11349743012245596, P83=0.27292290777614536, P84=0.8085338003013052, P85=1.984918864925328, P86=1.481424879444946, P87=0.6499165474526216, P88=0.3002375936495041, P89=0.27426873312551625, P90=0.3592714451224445, P91=0.4219270233574127, P92=0.15241774782036824, P93=0.08218474459278079, P94=0.23775760395214948, P95=0.3559664922912413, P96=0.3060103547000421, P97=0.7014301561410394, P98=0.6046570076008646, P99=0.1759229957398154, P100=0.0376754863231882, P101=0.047255518651621364, P102=0.2003432866986972, P103=0.19081795242890692, P104=0.045509173382138716, P105=0.036743666460178244, P106=0.06321383741801094, P107=0.15043770643960339, P108=0.10558500415115724, P109=0.03409693515838032, P110=0.023343551853229918, P111=0.054561296172297775, P112=0.09902822846991954, P113=0.2361273232306396, P114=0.11675295012680449, P115=0.021030927058366004, P116=0.008715155292154712, P117=0.010416773781649364, P118=0.04308700784176072, P119=0.1172084365347494, P120=0.06008330916288188, P121=0.0053523210167540506, P122=0.012610147907367805, P123=0.08223004400337841, P124=0.03660823755262414, P125=0.014809409013083351, P126=0.014828125695377167, P127=0.038530378229054904, P128=0.034508922233867735, P129=0.01301595331402791, P130=0.011788349868886544, P131=0.026386826248080798, P132=0.036499881149336215, P133=0.021568344710763812, P134=0.017010523090992893, P135=0.028503700347554218, P136=0.010578036072325513, P137=0.007480149708281622, P138=0.010255237150096144, P139=0.005015111759988949, P140=0.009648525617203833, P141=0.008100520663056407, P142=0.02414899817465957, P143=0.012312980495827072, P144=0.004995461792116991, P145=0.004354109153690088, P146=0.0013380316536719826, P147=0.0013985164733151484, P148=0.001474228724544135, P149=0.0028988441599947764, P150=0.004635787003232021, P151=0.00450016876106968, P152=0.00294709142463644, P153=0.0016583757652845897, P154=0.0021286964525481728, P155=0.0023878847627006785, P156=0.002900300548413166, P157=0.0042721158547882904, P158=0.004180533255030088, P159=0.003727045485837514, P160=0.0027090323205916306, P161=0.0022638926911404565, P162=0.0015111488234222563, P163=0.0014851887914485316, P164=0.001505325402355459, P165=0.0005807745499716334, P166=0.0013945422390895226, P167=0.002054777407371763, P168=0.001770024037281554, P169=0.0015173836393911008, P170=0.001233870851215816, P171=0.0008487579493498413, P172=0.0011071579840040322, P173=0.0029020676371658417, P174=0.004617898807283089, P175=0.0036005977849390336, P176=0.0021818657639378456, P177=0.0023043143785856385, P178=0.0014811599888937394, P179=9.348457610011153e-05),Row( P0=-795.8387776430202, P1=75.27136363927765, P2=16.483601993130893, P3=13.496508117227094, P4=-4.212958884863835, P5=4.980862586301141, P6=6.790160183938002, P7=7.461413585400546, P8=4.98113931821478, P9=-3.552958898043163, P10=-0.6878459379560983, P11=5.7682418795053145, P12=-0.1731316056713382, P13=0.7898972283497508, P14=-3.092314502772556, P15=3.297479759308565, P16=0.8035924240200963, P17=-1.0901509336892021, P18=-0.29440426124372154, P19=1.486864104631457, P20=-4.842131393439068, P21=0.8110795419591708, P22=-2.5080941552223015, P23=0.15537417488614907, P24=-0.3430708013735363, P25=1.332848316047448, P26=-2.2440900749398596, P27=2.357570462248486, P28=-3.44742082324161, P29=-0.4757384976571771, P30=-2.365625073498766, P31=1.7893652722538005, P32=-0.5566840271905181, P33=6.557407060377759, P34=2.380087214105629, P35=3.973158291810757, P36=6.094081077488482, P37=7.5572178170514, P38=2.652825491576338, P39=4.69040048473084, P40=0.675432144502195, P41=0.6626571900317428, P42=0.6706277183405024, P43=0.66955038362773, P44=0.7440959393401572, P45=0.7662350909445739, P46=0.6912813126926393, P47=0.6668411217954937, P48=0.7165437441935275, P49=0.766955954012125, P50=0.7890811342394533, P51=0.7394537379275695, P52=2.213509029602948e-05, P53=7.301666368094834e-06, P54=7.6462469501654e-06, P55=2.330536388736104e-05, P56=0.00013125524739821825, P57=0.0013470069093425216, P58=0.0028664765940741503, P59=0.001969932600966188, P60=0.00025083144500334865, P61=0.00010838370772219653, P62=0.00014559393826345813, P63=0.00021473235278970276, P64=0.00045255772788682245, P65=0.002534129362935766, P66=0.0031493063973321144, P67=0.0007655535160358966, P68=0.00015025436205659353, P69=0.00019738316618246062, P70=0.0002956273532623778, P71=0.0003470979246716013, P72=0.0007664190168901567, P73=0.0011427977628455399, P74=0.0006312671582572832, P75=0.0002066399710615325, P76=0.0001556333360453687, P77=0.00022834115885866387, P78=0.000752844904762326, P79=0.0012340685492255356, P80=0.0009732228619359796, P81=0.0006819749508275822, P82=0.0006131546396348157, P83=0.00019596064718946737, P84=0.000126592981016459, P85=0.0003159583680585334, P86=0.00044344752891506666, P87=0.0007399179153965109, P88=0.0005614821560610181, P89=0.0001762452764129672, P90=6.360349856608263e-05, P91=0.00013236062019753768, P92=0.0014275551785796036, P93=0.0007983794469671345, P94=0.00017168111790394632, P95=4.0133402625782144e-05, P96=0.00012519658038111025, P97=0.0006986559010150523, P98=0.00039942885724259994, P99=0.00012395529926742334, P100=5.723413094026175e-05, P101=0.0002063967775698304, P102=6.798590527562789e-05, P103=2.1747622774608942e-05, P104=2.6552671355144267e-05, P105=3.796986988521081e-05, P106=2.8872512129633203e-05, P107=2.314286494438933e-05, P108=4.3028027057087116e-05, P109=1.481763786420461e-05, P110=1.5418987304787168e-05, P111=3.358933258003631e-05, P112=7.2924693702781255e-06, P113=1.5927788907102356e-05, P114=7.3275951453406025e-06, P115=5.92605833296916e-06, P116=9.326311058895559e-06, P117=1.644720373768174e-05, P118=2.028914754258109e-05, P119=8.475684077453093e-06, P120=9.3245022826943e-06, P121=7.901425165004654e-06, P122=8.185064535268092e-06, P123=4.103388263389915e-06, P124=5.0554607818647606e-06, P125=3.787244979016136e-06, P126=1.5551715198096996e-06, P127=1.8559497771657736e-06, P128=1.6699801995438457e-06, P129=1.4292443486640086e-06, P130=1.5627904389529147e-06, P131=1.2957735698373085e-06, P132=5.541294066109635e-07, P133=8.189260529345608e-07, P134=1.7837546014387952e-06, P135=2.292698966913651e-06, P136=5.9344854885427e-06, P137=3.528146411659752e-06, P138=1.3974287516571518e-06, P139=1.0644369588904685e-06, P140=5.181641035065933e-07, P141=6.389953750261837e-07, P142=9.851996428129287e-07, P143=1.3885859515343182e-06, P144=3.115662595429984e-06, P145=3.3185963234085106e-06, P146=4.586663533872841e-06, P147=3.3163368355054765e-06, P148=4.9091584210944994e-06, P149=6.959549583207261e-06, P150=8.95950087642621e-06, P151=1.0064601784965285e-05, P152=1.035659488342033e-05, P153=1.2016240090674868e-05, P154=1.2827069624028437e-05, P155=2.0451275353679437e-05, P156=1.7035143417167582e-05, P157=1.923786233773e-05, P158=8.714292877500685e-06, P159=7.19446623332909e-06, P160=5.439830933178977e-06, P161=3.47771368961958e-06, P162=1.6134454175053799e-06, P163=1.123057119667691e-06, P164=1.1663051732079946e-06, P165=1.4274713951493897e-06, P166=1.3627590218278961e-06, P167=1.1871862186745044e-06, P168=7.371239367847299e-07, P169=6.340200473671774e-07, P170=6.994872793454969e-07, P171=7.884928133977828e-07, P172=9.555343374685277e-07, P173=1.3564968474593557e-06, P174=1.0477894418314891e-06, P175=1.0072466349439852e-06, P176=8.739585635434327e-07, P177=6.00193633201837e-07, P178=2.033210211257719e-07, P179=1.8573767538556604e-08),Row( P0=-605.5764664037771, P1=48.317141321988295, P2=-8.257264535168428, P3=-2.1599832381058772, P4=-17.22278252856746, P5=-6.81787291098871, P6=0.4763030436431375, P7=-3.397672083458614, P8=-2.5903783667383897, P9=-1.7498645849309429, P10=-2.1226595875058276, P11=-3.9002245662690447, P12=-2.53782023018036, P13=0.17833095941058194, P14=-3.8980840221877875, P15=2.012190059618317, P16=-1.0486862198113098, P17=0.643116157823422, P18=4.291303339770802, P19=4.173311422260207, P20=2.7897002058595888, P21=1.6745543189007368, P22=2.2616850859188022, P23=4.525541327669664, P24=-1.3398942986067526, P25=3.351785907866561, P26=0.9145875561977674, P27=3.0734328620947187, P28=2.2829911161355416, P29=1.9779274840938472, P30=1.5106252031627545, P31=5.020264010655486, P32=2.002184575111907, P33=3.289550197178226, P34=1.4868376654199063, P35=1.8806832466023973, P36=0.9306152017214218, P37=2.187241682053136, P38=0.5086119208117936, P39=1.8498464634799394, P40=0.697481849644868, P41=0.70975909249255, P42=0.7309141351556333, P43=0.7474131302590268, P44=0.7302130370459191, P45=0.73391535405302, P46=0.7155071233216816, P47=0.6981284515388351, P48=0.6941509286242956, P49=0.7058519822432706, P50=0.709626393418584, P51=0.6769982736460352, P52=0.0008101920424415756, P53=9.614237832492066e-05, P54=9.92694452974667e-06, P55=2.761246007587509e-05, P56=7.258644195791475e-05, P57=0.00029886708071554, P58=0.0008198399956192719, P59=0.0023591781556977372, P60=0.004035714693373563, P61=0.0037173867470280462, P62=0.008366735539125968, P63=0.013428833333955918, P64=0.010730213032192435, P65=0.01449582118495299, P66=0.009938560159612904, P67=0.009658206378277974, P68=0.008143214382888507, P69=0.01082887102863276, P70=0.01573159962731287, P71=0.0061818216483912935, P72=0.0025335702212842515, P73=0.0050248287590624685, P74=0.006732303859903993, P75=0.008057668317457294, P76=0.007075731197783434, P77=0.01472107423850743, P78=0.06861371369255734, P79=0.21699142835299792, P80=0.20266818163628172, P81=0.04187312526437113, P82=0.016389421269468612, P83=0.016705487249104728, P84=0.014778940759133184, P85=0.010210432637627317, P86=0.00933500032790011, P87=0.011476979211485968, P88=0.008319677724840088, P89=0.011690225633631186, P90=0.019279459824179553, P91=0.02808109695219381, P92=0.021945948691143764, P93=0.020180771872458432, P94=0.020449749914283066, P95=0.007372899290276356, P96=0.0072201729657685905, P97=0.00823518421738769, P98=0.014537877959814642, P99=0.009430162575267766, P100=0.001865441791756381, P101=0.002348764289279772, P102=0.0034929368676545197, P103=0.0025297235321540966, P104=0.004140102350573712, P105=0.0055432937515417086, P106=0.0031432814438857263, P107=0.004341232566170486, P108=0.004359683083976506, P109=0.0029739264219240716, P110=0.0013978392758151794, P111=0.0015746427770937166, P112=0.0008420962474864816, P113=0.00031959615387931273, P114=0.0002808763574789237, P115=0.0001414986378289144, P116=9.76140693663122e-05, P117=0.00039241975868338735, P118=0.00024058538910226357, P119=0.00025500826506918045, P120=0.00017019132660753796, P121=0.0001411212668664313, P122=0.0006044569535949859, P123=0.00042404393642136003, P124=0.00011521168609428831, P125=0.0002653111756616636, P126=0.00046407050727490244, P127=0.0005732517059183855, P128=0.00183033356492147, P129=0.0014955605524301896, P130=0.00032532413110616926, P131=0.0007831168499569114, P132=0.0017319638716712172, P133=0.0005302831837241299, P134=0.0005426238176610565, P135=0.000269077640199025, P136=0.0003790975509813355, P137=0.00041568284857146847, P138=0.00023693883270732233, P139=0.00017155752425706395, P140=0.00022885056383949325, P141=0.00014704495893330357, P142=0.00024323922929092438, P143=0.00019421362479159912, P144=0.00030166888990697493, P145=0.00021318481375781588, P146=0.0001431597631883839, P147=9.378066687846024e-05, P148=0.00033099417392081073, P149=0.0005751650512193069, P150=0.000608337401662309, P151=0.0007354036978765713, P152=0.0006385646671149993, P153=0.0007464457592977492, P154=0.0005090644573701702, P155=0.0003180419580257343, P156=0.00031165102117378134, P157=0.0002748631656455666, P158=0.00013293468356086432, P159=0.00010929443145643552, P160=0.00011026296626429377, P161=0.0001694742721232004, P162=0.00023971225564981935, P163=0.00032944556765655847, P164=0.0002532359646880398, P165=0.00020771335047236955, P166=0.00012715533264024507, P167=0.00013714405730090687, P168=0.00014934347849062784, P169=6.771091872345903e-05, P170=7.465239220243332e-05, P171=9.443493510535178e-05, P172=0.0001065717503663377, P173=0.00010632914651167598, P174=0.00010720030339662865, P175=9.19852564834511e-05, P176=7.836621422029077e-05, P177=7.161644748004004e-05, P178=3.355863606931535e-05, P179=3.5988706378548245e-06),Row( P0=-672.3449188380889, P1=63.94956539638597, P2=-6.483946978664198, P3=10.599634237369147, P4=-3.6807897495807085, P5=-6.371528745331435, P6=0.8222552755863357, P7=0.45251932500437336, P8=-0.683132371373303, P9=1.0379095187199072, P10=-5.145299052633731, P11=-0.4147523865727448, P12=1.5047745665680634, P13=-3.6292155673031194, P14=-1.2413043910528465, P15=1.3511041220275586, P16=-0.6229268859415029, P17=2.381533222707818, P18=-1.0950913961033069, P19=2.429521340714944, P20=-0.6331406321223194, P21=0.013469784908408299, P22=3.4013112451703797, P23=1.6334510907133049, P24=-0.16123472176000705, P25=2.794936240613336, P26=-0.5756133239175737, P27=3.433741249368973, P28=1.9870299916650307, P29=0.7081339528638523, P30=0.260428123825102, P31=2.1365843433649476, P32=-1.2023614470188086, P33=2.581902731869173, P34=-1.048962097884654, P35=2.6244464607962494, P36=0.2424569755596332, P37=2.748805638141658, P38=2.78605300746163, P39=2.1745184363294703, P40=0.6791986940839075, P41=0.6870405610142425, P42=0.6754671096295762, P43=0.700434643354588, P44=0.7336679487986422, P45=0.7850883047425055, P46=0.7676886256507837, P47=0.7742668316776192, P48=0.7697046029836565, P49=0.7440046798753467, P50=0.735611391080015, P51=0.7031308990675063, P52=0.00026315221942551124, P53=4.006959572053738e-05, P54=1.2283872639698141e-05, P55=4.989807517204849e-05, P56=0.00045323457428114364, P57=0.0030545669890430513, P58=0.0037738302079356936, P59=0.0034357197812645854, P60=0.005962507760833285, P61=0.00880550487740905, P62=0.01818841225829935, P63=0.025448946675280307, P64=0.009487008434867512, P65=0.010912744949186415, P66=0.011842490348560877, P67=0.004644021788019893, P68=0.006865028781614526, P69=0.00925468703558042, P70=0.009310706641048565, P71=0.007506640098373734, P72=0.012825312584721034, P73=0.023215000384862027, P74=0.021623174689321935, P75=0.043567913102905774, P76=0.05198805162280655, P77=0.02122851488206981, P78=0.014015551449656176, P79=0.016951468323036272, P80=0.01190360950045912, P81=0.004684890494665813, P82=0.004262994737687087, P83=0.003984290740952359, P84=0.005916700505741924, P85=0.006257562416456986, P86=0.003459879025928114, P87=0.005488687642816417, P88=0.007959876106337334, P89=0.007293241854943828, P90=0.0055428055081214464, P91=0.004487721495941821, P92=0.012289985328155345, P93=0.011475113488817622, P94=0.009179696200711206, P95=0.011062308337042304, P96=0.006688457569156772, P97=0.01687554883585856, P98=0.01339651437676327, P99=0.005775313565939643, P100=0.0018716157786651216, P101=0.0016345880459660122, P102=0.0018510579465416542, P103=0.0004499903263797704, P104=0.00040926778637741674, P105=0.0013518182890924844, P106=0.004732728589695048, P107=0.0028939242397199296, P108=0.0016779585476124567, P109=0.0017819552228643576, P110=0.004435679865518244, P111=0.005579276441441863, P112=0.002964776811165634, P113=0.0014276759296605812, P114=0.0006031217179472934, P115=0.00035955501426378824, P116=0.0005535430598859932, P117=0.0006610503547853252, P118=0.0007162863705893706, P119=0.0007753365255459662, P120=0.0008786163024490665, P121=0.00119251839248687, P122=0.0003201581279203866, P123=0.00033634195381876755, P124=0.001233822763180513, P125=0.0002928805525121807, P126=0.00017876954371938723, P127=0.0011631567810944739, P128=0.00037521048002808006, P129=0.00011865163111605857, P130=0.00023964435882526663, P131=0.00010853792120861503, P132=0.00033492734461262874, P133=0.0003014923264031803, P134=0.0006557925674705289, P135=0.0005550238641547142, P136=0.0003555674922549978, P137=0.00025112025946202925, P138=0.00010699239096254539, P139=0.0001510803490754248, P140=0.00011319687513894057, P141=0.000130383601886786, P142=0.00018132143525340016, P143=0.00015247116762912865, P144=0.0002128828557933357, P145=7.593683288047413e-05, P146=5.138943695019688e-05, P147=1.4947277996290971e-05, P148=5.631365294861817e-06, P149=4.317236716082421e-06, P150=8.094073080091992e-06, P151=1.712599538133059e-05, P152=1.3994410254474437e-05, P153=2.708856606234352e-05, P154=5.7964864048057505e-05, P155=5.625268800429699e-05, P156=3.3817416931908315e-05, P157=4.015916792680629e-05, P158=3.899512190745228e-05, P159=2.7430343875551342e-05, P160=2.827682198883242e-05, P161=3.291413156115994e-05, P162=2.534280552399549e-05, P163=1.0642894614996735e-05, P164=1.651506314958303e-05, P165=1.3063239676681552e-05, P166=8.854412362415624e-06, P167=1.3953360585269461e-05, P168=1.2797774498772536e-05, P169=1.0927239399533945e-05, P170=3.693425676963345e-06, P171=3.171419113367171e-06, P172=1.5842078689873069e-06, P173=4.788695275690959e-06, P174=5.205440921133641e-06, P175=7.160555819328263e-06, P176=5.9112300023855275e-06, P177=3.5507341780646907e-06, P178=1.4066482927824476e-06, P179=1.5926137745216308e-07),Row( P0=-519.0947663506554, P1=26.883295590639776, P2=-19.34860859735706, P3=-4.217836606111938, P4=-16.508683649317216, P5=-15.566578457841114, P6=-7.527000141008827, P7=-5.565564494576355, P8=-7.79983129692541, P9=-2.2000959976944925, P10=-9.089274022565217, P11=-3.7187338986771823, P12=6.177204053323796, P13=4.997593609114348, P14=9.188081223569114, P15=14.09731477539193, P16=8.536007182717277, P17=9.346401069086534, P18=2.6283882345244316, P19=5.503484979203034, P20=4.004590002256007, P21=4.50393523970216, P22=7.527317134003114, P23=3.6678734900565293, P24=-0.49851744378990137, P25=1.8217798625911676, P26=1.8941422557161274, P27=2.8695123824767057, P28=2.465054070394513, P29=4.388685288932269, P30=-0.9511164465242307, P31=4.527328016931804, P32=0.028869979689512045, P33=1.6269362636585274, P34=0.6063054483966748, P35=0.7670757190389926, P36=1.5965298590644204, P37=2.449152796090838, P38=2.4033897156549666, P39=-0.26227033482917667, P40=0.5602137405511656, P41=0.6211688055196742, P42=0.6422071718238032, P43=0.5957376398266029, P44=0.582839794773722, P45=0.6547647465511878, P46=0.6459821885716466, P47=0.6407975992510487, P48=0.6394882458214213, P49=0.6032667788762605, P50=0.602357408990583, P51=0.5917231299133714, P52=0.0011123699371147404, P53=0.00014933976657031256, P54=2.4061423166271578e-05, P55=3.498703870627732e-05, P56=3.409403129532629e-05, P57=4.538807629219681e-05, P58=2.5757216291530866e-05, P59=1.8286196770402212e-05, P60=4.1043583617927504e-05, P61=0.00022003608987860513, P62=0.015743856988217654, P63=0.053797327784666046, P64=0.02780817472789698, P65=0.033271551441447715, P66=0.07092849903598128, P67=0.12259934649075983, P68=0.3431724909922385, P69=0.6260462954608723, P70=1.2439692061478043, P71=0.825078272472358, P72=0.1996498289141397, P73=0.13903163658111675, P74=0.07113979731311182, P75=0.031784835642826775, P76=0.018337464934406696, P77=0.01763509799231333, P78=0.03371844202019743, P79=0.06634578686773898, P80=0.08220082663236601, P81=0.10131666782029061, P82=0.17760720045450912, P83=0.35139510339792057, P84=0.6605708313636433, P85=1.6666105072132569, P86=2.066607026265654, P87=1.0377499584286214, P88=0.6646053669109655, P89=0.6529438496587763, P90=0.8102602603375875, P91=1.2568025929028084, P92=0.6839058805057485, P93=0.27594017492304324, P94=0.09057728008343975, P95=0.09602578411909608, P96=0.13582568217197064, P97=0.4670115122703779, P98=0.6874165978626218, P99=0.3360473090245918, P100=0.18588693026109382, P101=0.1744214826926234, P102=0.10347072569260285, P103=0.0805376375688777, P104=0.03402965109970959, P105=0.03966751474461512, P106=0.05118369495020104, P107=0.11529324401488956, P108=0.19000844310898887, P109=0.13002931494964615, P110=0.08818789125648892, P111=0.22760043744143676, P112=0.6975316736691992, P113=0.2776960281995626, P114=0.07638288836950302, P115=0.04672507789024559, P116=0.017776818601517804, P117=0.019409506328380843, P118=0.07258078855826751, P119=0.1119001250845437, P120=0.05170355220927466, P121=0.005337448899598236, P122=0.008525238983047112, P123=0.05153340624708713, P124=0.021313509241925266, P125=0.007642750429341402, P126=0.003536660858816717, P127=0.01912359722295013, P128=0.017239133371507606, P129=0.007308620216672449, P130=0.00920285289461647, P131=0.02638050768407118, P132=0.026196209010052533, P133=0.03708866768009541, P134=0.03703495427078739, P135=0.03081047663370902, P136=0.010806620261417139, P137=0.011470824456435425, P138=0.012963498792396617, P139=0.011115021488637243, P140=0.015004567845425523, P141=0.014268778374839175, P142=0.016179827556139274, P143=0.009032272709520615, P144=0.007185934820597419, P145=0.001563140727964397, P146=0.001220021243427299, P147=0.0013283824823942237, P148=0.0014356359470172597, P149=0.0022904543204985547, P150=0.0016813307527921754, P151=0.0020368238686096552, P152=0.0016438341960845744, P153=0.0017501420875579334, P154=0.0015644391569731228, P155=0.0017781834752070254, P156=0.003053782242446738, P157=0.0038504824808272275, P158=0.00344075831478585, P159=0.0023043065926759356, P160=0.001700994510349257, P161=0.0009881688012795562, P162=0.00045589610833408553, P163=0.0006401329848067346, P164=0.0014177476231876564, P165=0.0017878353668664487, P166=0.001639564974390838, P167=0.0018579935781293173, P168=0.0016600076218045237, P169=0.001359834193250314, P170=0.0018275814442122098, P171=0.0023612942146641193, P172=0.0020055145819768555, P173=0.003430611770929133, P174=0.002324340457854303, P175=0.001878877554271274, P176=0.0022383978800892814, P177=0.0012674155115548935, P178=0.0004778678013116786, P179=3.8472490226395455e-05),Row( P0=-684.34389189838, P1=54.33424322144201, P2=-3.054943707677764, P3=4.3441609748224685, P4=-8.036126341004259, P5=-7.27872160305381, P6=1.0867146220733963, P7=-0.5960537502475324, P8=-1.189356554147351, P9=-1.515223614782662, P10=-1.3532536775762738, P11=0.16686071652663095, P12=-1.0156841028042958, P13=-1.3588930785433229, P14=-3.5374866618786625, P15=1.9487999797452658, P16=1.6639078439063695, P17=2.4507927262717715, P18=2.055688390438984, P19=3.651098325223521, P20=1.631699782713977, P21=7.743663089518655, P22=4.7354847004170555, P23=6.892814536265031, P24=2.9614759715664993, P25=4.798921694312099, P26=3.772455505012985, P27=4.872898553648824, P28=4.0213658072934075, P29=4.2252823534518456, P30=-0.554326518941874, P31=3.384208268150588, P32=-1.5950584470851574, P33=1.8817175927639058, P34=0.48259609683000115, P35=1.6809004046262643, P36=-0.7767888516094269, P37=0.14350145231788897, P38=-0.35628580926341147, P39=0.7766094969684487, P40=0.6635666701841971, P41=0.658851942274967, P42=0.6874606388880417, P43=0.7390005594774685, P44=0.7276088900105048, P45=0.7354397017672917, P46=0.7107450020003628, P47=0.700253246301119, P48=0.706648150493142, P49=0.6990354549330343, P50=0.714741318884419, P51=0.6973332152764766, P52=0.0002926017273330038, P53=6.506298520084856e-05, P54=1.5006616767569592e-05, P55=2.7501189357326568e-05, P56=2.5839270388511014e-05, P57=5.884939256468863e-05, P58=0.00031038184098560855, P59=0.0007421143167156477, P60=0.004516911489003963, P61=0.012124047673123707, P62=0.017313586505854087, P63=0.020939385170099425, P64=0.011291002175294374, P65=0.005613725068583409, P66=0.0018176749960973873, P67=0.0011773217591977518, P68=0.0012668674293874011, P69=0.004486770913800639, P70=0.011773334763807645, P71=0.011473048712609195, P72=0.007525550824883384, P73=0.007346398656755472, P74=0.009735420356790323, P75=0.01135279514684472, P76=0.00758898993256618, P77=0.008126038054620652, P78=0.01150361784948311, P79=0.01064480338833384, P80=0.009726327364208174, P81=0.015283471931738011, P82=0.02404148491738535, P83=0.017661325929283475, P84=0.01396203710345788, P85=0.02036266426194013, P86=0.01879769508135651, P87=0.010360291524676404, P88=0.019532992420839227, P89=0.012040593864897755, P90=0.004740811877303607, P91=0.00240433473388926, P92=0.0012930044351607112, P93=0.0010265062852616894, P94=0.002384308593774408, P95=0.004131744316540774, P96=0.028639644020384422, P97=0.04080288456988914, P98=0.017219546730598674, P99=0.006257289387827735, P100=0.0018936540777914305, P101=0.0011896962570320598, P102=0.0009098246300150885, P103=0.0014952683357607514, P104=0.003318344525397445, P105=0.0032731598689669395, P106=0.0010275102787746621, P107=0.0008245853944991557, P108=0.0029142480330475217, P109=0.00390508664086614, P110=0.000566595168434952, P111=0.0013489437441557253, P112=0.0003248644985086136, P113=0.00014117072757065854, P114=0.00021541475262806846, P115=0.00017143806270968797, P116=0.00011600794037370184, P117=0.00011323238479477787, P118=0.0001861684962569859, P119=0.0003926689096578277, P120=5.335420370503166e-05, P121=4.923987158309612e-05, P122=0.00011235890991127271, P123=5.1463823986124536e-05, P124=5.606198712505847e-05, P125=6.384910756408999e-05, P126=4.392172238889569e-05, P127=7.796772202754608e-05, P128=8.553877807008387e-05, P129=3.780366431168853e-05, P130=5.773624369135965e-05, P131=0.00010446553738685355, P132=0.00013440670370467808, P133=0.00026345693770465933, P134=0.00033227253770403017, P135=0.00035246719400638054, P136=0.0001306499034392278, P137=0.00018233086418374537, P138=0.00011996189981658027, P139=0.00015326376454697456, P140=0.00015358351903478132, P141=0.00010060939891464331, P142=8.481331925007978e-05, P143=4.9387327208131954e-05, P144=3.0397939019733997e-05, P145=2.6950679835255176e-05, P146=3.441613768302401e-05, P147=6.486399475347372e-05, P148=2.7185136168812762e-05, P149=5.15984295715275e-05, P150=0.00017362325315990504, P151=0.00015641307763541773, P152=8.005748913847687e-05, P153=5.35575667083198e-05, P154=6.306195032763463e-05, P155=2.9610462972253636e-05, P156=1.6541601450301933e-05, P157=3.202496767046161e-05, P158=3.1317476813923236e-05, P159=2.6941635694455244e-05, P160=2.8227115296135133e-05, P161=2.685152935318494e-05, P162=1.819083471147394e-05, P163=1.3017276513074905e-05, P164=9.309685213233406e-06, P165=8.530518196367548e-06, P166=1.4022738843546564e-05, P167=9.217679729158019e-06, P168=1.3845637395775954e-05, P169=1.1874369115206577e-05, P170=1.3615941502684457e-05, P171=9.527681170450322e-06, P172=9.237004183658346e-06, P173=1.512321563363193e-05, P174=2.72493577621594e-05, P175=1.8849393504367463e-05, P176=2.4298816475390922e-05, P177=1.717945081614682e-05, P178=8.570467044768476e-06, P179=1.2922929698620565e-06),Row( P0=-743.7147756246546, P1=74.57052312493828, P2=3.027556185076585, P3=13.268864076935595, P4=-3.419986156026658, P5=4.195487864613239, P6=5.837281165446739, P7=6.596531884002706, P8=1.2162610257071536, P9=-0.6229545949260751, P10=-0.5601302965572417, P11=1.2306174094287352, P12=1.5168336537068106, P13=-1.3184110894122554, P14=-3.57442950430586, P15=2.5313995279546373, P16=-0.11456572934266233, P17=1.9399758333241979, P18=0.3063126406116921, P19=1.2917127474031476, P20=-3.8408404357749917, P21=-0.5050949550203684, P22=-2.3515208175627844, P23=-0.21914129995631768, P24=-2.283399590393083, P25=1.7893045924796487, P26=-2.2044292883524714, P27=4.704104133460693, P28=-1.420797119431248, P29=0.9570304834759417, P30=0.9096982419751638, P31=3.83510315339151, P32=2.3828233555901557, P33=7.002718135445678, P34=1.2990144136699004, P35=1.8539110320611387, P36=2.6062878970821237, P37=2.049728667299674, P38=1.2915170923924135, P39=2.2793844873444207, P40=0.7146514158736851, P41=0.6707056312655693, P42=0.6816846261953421, P43=0.6931280580379954, P44=0.6939747924891799, P45=0.7151428346901395, P46=0.736732789432709, P47=0.7293339400892337, P48=0.7209515879247527, P49=0.7232289887185681, P50=0.7228238560374449, P51=0.728998981991907, P52=0.00043792262519438876, P53=3.877400707670829e-05, P54=1.628687499859832e-05, P55=5.430505826891999e-05, P56=0.00042919120321022544, P57=0.002651597006910191, P58=0.0026225768001017875, P59=0.00413566444965194, P60=0.00477737286866238, P61=0.004028194569902638, P62=0.0008497138101913858, P63=0.0004463040768688237, P64=0.0002925634780474699, P65=0.0002532737841315981, P66=0.0013341122786867576, P67=0.0028041217984408843, P68=0.0012337588827317353, P69=0.0016732909855649905, P70=0.004570248430651522, P71=0.0018144302755896077, P72=9.712613639786967e-05, P73=0.00010875776009294571, P74=0.00020175488577684297, P75=0.0002686769238754851, P76=0.00019774378896364396, P77=0.00031079525971502796, P78=0.00039047436541427325, P79=0.0019686203315351872, P80=0.0024053797190894915, P81=0.0007699820663445661, P82=0.00036347076706563097, P83=0.00024025589471424012, P84=0.0003464233258812774, P85=0.00040008825756215205, P86=0.00012292487354243246, P87=5.642477448054392e-05, P88=0.00014024967206621833, P89=0.00013351874616780395, P90=0.00015326067496690456, P91=0.0001891024672950537, P92=0.00022434984975289228, P93=0.000121351104784335, P94=0.0001964662497577866, P95=0.00019857079205026963, P96=0.00019807499456207624, P97=0.00020447344359922402, P98=0.00020874221023156955, P99=0.00029694594930521767, P100=0.00022220802713668195, P101=8.682191752289536e-05, P102=5.093877881779143e-05, P103=6.59181804645023e-05, P104=3.5272686313720804e-05, P105=2.6774466279477094e-05, P106=6.277479040556083e-05, P107=9.148403031747017e-05, P108=4.9606002534476134e-05, P109=1.9855873586548195e-05, P110=2.361709200908477e-05, P111=8.092837273232024e-05, P112=3.8875952486088905e-05, P113=2.0448318566997547e-05, P114=1.5823899382511107e-05, P115=1.6589542901416856e-05, P116=1.3742010062248999e-05, P117=2.647696284935244e-05, P118=3.303659179037325e-05, P119=1.5033775230252274e-05, P120=2.3492665833966245e-05, P121=3.0233432914634828e-05, P122=1.9271657876479973e-05, P123=1.2838402287899983e-05, P124=1.37706151785151e-05, P125=1.1159993633559436e-05, P126=1.403167837385024e-05, P127=1.6036680989751616e-05, P128=1.2009109986499876e-05, P129=1.537689167062102e-05, P130=2.5204357937168862e-05, P131=2.564786130856826e-05, P132=9.46259570599993e-06, P133=8.397552989352988e-06, P134=1.7536468661857836e-05, P135=1.5272862454578683e-05, P136=2.6388540975939316e-05, P137=2.4932103668274907e-05, P138=2.229056854359095e-05, P139=2.7752890848082088e-05, P140=1.0916349731937709e-05, P141=9.154570960147802e-06, P142=4.552613270275067e-06, P143=4.659408073349033e-06, P144=9.374841288308103e-06, P145=8.771117626539964e-06, P146=1.1562766672642827e-05, P147=5.64321756307879e-06, P148=5.870417595232013e-06, P149=5.826614552617484e-06, P150=8.284751628060199e-06, P151=1.4214921315051161e-05, P152=1.753263962588172e-05, P153=2.1443328943823783e-05, P154=2.2273309443036694e-05, P155=2.3147586996907896e-05, P156=3.406602158359169e-05, P157=3.5763292762907195e-05, P158=4.231485815059882e-05, P159=0.00011994629642927465, P160=0.00020018618771838357, P161=9.348391940230465e-05, P162=5.0489215436333436e-05, P163=1.8126308337931263e-05, P164=1.0186590883796872e-05, P165=1.882844788632678e-05, P166=1.1788712548695352e-05, P167=7.351736288512197e-06, P168=5.779368912440086e-06, P169=2.880707612038089e-06, P170=1.9445645125286307e-06, P171=2.006519088644538e-06, P172=2.3588277977843492e-06, P173=2.7827679960615444e-06, P174=2.673476696829786e-06, P175=1.7343745051475502e-06, P176=9.46177620314681e-07, P177=5.207564786570181e-07, P178=1.9715737319740926e-07, P179=2.4091358511342596e-08),Row( P0=-638.3730480306131, P1=52.30854214892795, P2=-8.824161948059979, P3=-4.309553994072485, P4=-13.721093416897101, P5=-7.346974244831362, P6=-7.491799087979945, P7=-1.993215202170396, P8=-2.335491376635863, P9=0.25734493926813357, P10=-6.325791593905103, P11=-3.7893200638903077, P12=5.101203962507966, P13=1.0971476505030169, P14=9.010894939509395, P15=13.442630820088574, P16=7.910659761385745, P17=8.682499134576249, P18=7.407410102978785, P19=7.211927597742295, P20=2.5038681514659173, P21=4.1669532776552245, P22=2.928781700393859, P23=4.1276493422777385, P24=-0.6657698016022192, P25=3.165718650906823, P26=1.7862601880533566, P27=3.6710101706934886, P28=1.4642739099705362, P29=1.55854597156741, P30=-0.5154482773767344, P31=2.470696987672237, P32=-1.606432268022493, P33=2.0587087479918167, P34=-1.2013484997948725, P35=1.8052214484252993, P36=-0.12277008293204313, P37=0.4956885093480246, P38=1.2067557878774453, P39=2.7767131273627492, P40=0.6574942304670843, P41=0.6590094752330758, P42=0.7094536488021722, P43=0.7013413913796699, P44=0.6541847466334615, P45=0.6314384254162512, P46=0.6310779817073786, P47=0.6167637656152857, P48=0.634138597358895, P49=0.6458279176318288, P50=0.6304955328395294, P51=0.6566059944405451, P52=0.000946708100805053, P53=0.0001400629531113338, P54=3.302314747100008e-05, P55=4.0595784447246676e-05, P56=4.2590842023586006e-05, P57=7.083170242100812e-05, P58=5.250000732883792e-05, P59=8.017741786469088e-05, P60=0.00022935764164555957, P61=0.0005432222587482262, P62=0.0015167076231529463, P63=0.003508265068786639, P64=0.007430161929209077, P65=0.024169640308940955, P66=0.02631608413948602, P67=0.023429086022208365, P68=0.024677967586112702, P69=0.044566587991532805, P70=0.0554360108732621, P71=0.025031579154017606, P72=0.0036635746310186112, P73=0.001337689655462595, P74=0.000782508532948581, P75=0.0006456639565372322, P76=0.0006330723744583861, P77=0.0024983327806177113, P78=0.010773592148523348, P79=0.016000535995901843, P80=0.010219887145895393, P81=0.014636594504046833, P82=0.06777585970909655, P83=0.08311834901339883, P84=0.03583625545660886, P85=0.019080276182940885, P86=0.008612214599453778, P87=0.005922207851195803, P88=0.010998197906096152, P89=0.010634889620809123, P90=0.008157181228043833, P91=0.0061782830801820595, P92=0.0039150151018509305, P93=0.006345591954729359, P94=0.009169226121339776, P95=0.01169247651969338, P96=0.006234058820806403, P97=0.006058839172813857, P98=0.005190669228740042, P99=0.0019718330675462794, P100=0.0009163323620046933, P101=0.0011318895544580542, P102=0.0011498894173042414, P103=0.001507461035324454, P104=0.002273233647228758, P105=0.0047811444694506955, P106=0.01386461763321816, P107=0.011390658110949909, P108=0.003999501450006379, P109=0.0020979639064620044, P110=0.003057541057204283, P111=0.004845857641003114, P112=0.001424700818168185, P113=0.0004544175933001335, P114=0.00021105956170541117, P115=0.00035306698595446924, P116=0.00047313502557413554, P117=0.0005154071767018212, P118=0.00024848056237611413, P119=0.00014634079447870758, P120=0.00016600301011962344, P121=0.0005931934760115816, P122=0.00024789501417702114, P123=0.00010517454221110611, P124=8.284056724182765e-05, P125=8.845382540566095e-05, P126=5.9284492520574e-05, P127=3.173865347109729e-05, P128=2.7615251476480443e-05, P129=5.170964615046702e-05, P130=8.086007590376898e-05, P131=0.0001190490561397539, P132=0.00013594117781052736, P133=0.00015822335364979828, P134=0.0001721618106014341, P135=0.0001686332129033906, P136=0.00021800119555538103, P137=0.00010384515673169866, P138=0.00014705862591369673, P139=0.000171476908199574, P140=0.00012429253063038856, P141=8.577780687148625e-05, P142=9.062898880192893e-05, P143=9.955430064286111e-05, P144=6.357626040076e-05, P145=4.1457618142948374e-05, P146=2.9817070518557752e-05, P147=2.1183289528289043e-05, P148=1.840296999964625e-05, P149=5.3528753312206505e-05, P150=5.5072790095333014e-05, P151=0.0001493125675698148, P152=0.00021902289073412427, P153=0.00021437075262186806, P154=0.00016575104190188206, P155=8.265452154808682e-05, P156=0.00010463213401089397, P157=5.839661613898934e-05, P158=0.0001408389480805972, P159=0.00018876246972197963, P160=0.00021424798189657472, P161=0.00027863049213080885, P162=0.0002407069393618048, P163=9.459130640009547e-05, P164=7.678483365129347e-05, P165=0.000123307897917144, P166=9.58233917379249e-05, P167=8.744232064856706e-05, P168=5.9307616926108696e-05, P169=2.4153885676658645e-05, P170=1.4378114885915965e-05, P171=1.6362715137985465e-05, P172=2.273807363395988e-05, P173=2.658918783872063e-05, P174=2.343950696817504e-05, P175=2.003914099799633e-05, P176=2.4313532468658215e-05, P177=1.3624280075815081e-05, P178=6.187825891421098e-06, P179=6.805059599943859e-07),Row( P0=-683.8844473166145, P1=48.690455742903644, P2=1.2494472684287337, P3=5.793991935224168, P4=-5.87253354331668, P5=3.7931024325506932, P6=9.34290333942938, P7=4.551747296229461, P8=1.3492716784594365, P9=0.6916359195245034, P10=-1.5758244757777826, P11=-0.2033816684904937, P12=1.6579924212751906, P13=-1.7414747615333819, P14=-1.8332308844805631, P15=2.626083121148529, P16=1.7746035137364229, P17=1.6247995342752617, P18=3.0388362967240887, P19=1.456542620251991, P20=-1.7635467546964974, P21=-0.0873068272908243, P22=-1.7585564156568292, P23=1.4483765634667527, P24=-1.965345768599508, P25=2.1093638336291693, P26=1.019815810751603, P27=2.695388874103294, P28=-0.22649944326172872, P29=2.5190719162650517, P30=-0.963599731478689, P31=3.014972091001152, P32=-2.317892231384371, P33=0.9850237689012067, P34=1.1562489275051442, P35=-0.8414404133831941, P36=-0.23223289115877746, P37=0.21772361721212993, P38=0.9212002228249832, P39=1.551087957382793, P40=0.714890906348379, P41=0.7225057037384222, P42=0.7428606036279232, P43=0.7768360295929253, P44=0.7785070615145302, P45=0.7761254744845063, P46=0.8003291840788418, P47=0.780077694778666, P48=0.733496881090723, P49=0.7477343004672293, P50=0.7498878195589159, P51=0.7253938510031951, P52=0.00014449844118171915, P53=2.5599839059554573e-05, P54=8.90259856298801e-06, P55=3.0287937491767654e-05, P56=0.0003664414387805113, P57=0.0021143610739072197, P58=0.001370045980524366, P59=0.0011615648687032139, P60=0.004976448555373922, P61=0.009925342045439057, P62=0.004352679137911276, P63=0.0009765603753708889, P64=0.0005199413704048198, P65=0.0004366823020683262, P66=0.0006995474324633175, P67=0.0008728406701548109, P68=0.001745322495363724, P69=0.002941328028385936, P70=0.0051882103646125706, P71=0.013128789659366414, P72=0.006692387881399712, P73=0.0007770486921495303, P74=0.0001553325384970889, P75=0.00015318949252883578, P76=0.00024981079479587055, P77=0.00034729062178343826, P78=0.000688040101024066, P79=0.001724923127636225, P80=0.00788795267327158, P81=0.03874441751370173, P82=0.037509986295401816, P83=0.004797712585949833, P84=0.0006541048428949078, P85=0.000574657404677911, P86=0.00040051703853287944, P87=0.0005369239500825716, P88=0.002274889844252641, P89=0.0014658267175212787, P90=0.000994384318120144, P91=0.001165873963871693, P92=0.0012008779662178394, P93=0.0012890213166811533, P94=0.002303505333984977, P95=0.003866759216969342, P96=0.001600853107790622, P97=0.0011130671816266871, P98=0.0011189696255091393, P99=0.0014721930162140187, P100=0.0018391692871640708, P101=0.003780995953306687, P102=0.0010813858975525377, P103=0.0005965880307084754, P104=0.00112049532927548, P105=0.0014611317451812761, P106=0.0016614959117870714, P107=0.0005694899033153368, P108=0.00024004626970498979, P109=6.650109728008681e-05, P110=3.256395586031026e-05, P111=9.027559933800812e-05, P112=7.41974151009512e-05, P113=5.326482418594594e-05, P114=1.4260928470045561e-05, P115=1.980433302528234e-05, P116=1.783949773028581e-05, P117=4.408185755360394e-05, P118=0.00011093416049389608, P119=0.00010410280963157713, P120=3.446856595600929e-05, P121=0.00011105278129314724, P122=0.00010331357096026052, P123=5.981601056613926e-05, P124=0.0005844200530110053, P125=0.00011884386473065695, P126=0.0013311161895941297, P127=0.0009743075060804212, P128=0.0003673335560958451, P129=0.0003524752871678454, P130=0.00017898058084075312, P131=0.0004245219253919531, P132=0.00023229808223733104, P133=0.0005868032431376114, P134=0.0002430908321041479, P135=0.00017494539845734663, P136=0.00019373128744082957, P137=0.00013629095381084903, P138=0.00014013502783834984, P139=0.00010221289390476643, P140=6.394691955129112e-05, P141=9.728637690656961e-05, P142=9.542488368895071e-05, P143=0.0001212007281859741, P144=0.00016511007584432965, P145=5.770142657658385e-05, P146=3.437671586464383e-05, P147=3.839386232254296e-05, P148=4.9246934351115e-05, P149=0.0001624638376320276, P150=0.0002755437718855425, P151=0.000131087065801917, P152=6.06138502410118e-05, P153=6.329139265528786e-05, P154=4.9960861508354106e-05, P155=4.9929798061312926e-05, P156=9.227651475020155e-05, P157=0.00016264638800654623, P158=0.00032550169241843104, P159=0.00010068517745490491, P160=6.643789406534442e-05, P161=3.9507031741425236e-05, P162=2.9512508322594115e-05, P163=2.186742098448782e-05, P164=2.8924583267175616e-05, P165=3.2957906682494495e-05, P166=2.2584548622585955e-05, P167=2.1144375375849178e-05, P168=1.0581269096496887e-05, P169=1.1530073099910126e-05, P170=9.66794549197407e-06, P171=1.3050435902157128e-05, P172=1.7803819634695295e-05, P173=2.0651997624221256e-05, P174=3.990071164877941e-05, P175=4.709473319932488e-05, P176=5.212263553609974e-05, P177=3.811219392351612e-05, P178=1.548071399498672e-05, P179=1.462058987487281e-06),Row( P0=-720.5410042817986, P1=75.73952167340467, P2=16.52065336306857, P3=11.261627377040137, P4=-3.3183713526534127, P5=-1.468475207729466, P6=-3.481866143829985, P7=-0.7371376820711464, P8=6.076084262127382, P9=-2.669644979986121, P10=2.8789166150109313, P11=4.322216573196241, P12=3.179028968868841, P13=-0.8798140140952575, P14=3.3188893814580704, P15=-0.6398579601338064, P16=2.8989185732346803, P17=8.194367041932239, P18=9.588816162074528, P19=10.775008939357683, P20=8.162322475245452, P21=10.244503263730625, P22=5.602557108239916, P23=4.681877187625849, P24=0.7500758067023753, P25=1.3572723806645337, P26=0.6733784127554482, P27=0.23340830133346643, P28=-3.817325509845788, P29=0.5594199286893249, P30=-2.0352845055455333, P31=0.8080199008980076, P32=-1.24303822330589, P33=-1.3383728773599348, P34=-1.0042925296966048, P35=0.8606902922867661, P36=-0.8026236233454899, P37=0.8492315905532454, P38=-1.4323073667116124, P39=0.7913181645242731, P40=0.6178927718348183, P41=0.5966492222626167, P42=0.6135880215066913, P43=0.6041151408050064, P44=0.6393231502899033, P45=0.7041966887695137, P46=0.7591012229989513, P47=0.7338550127020007, P48=0.7735586715502429, P49=0.7361622896217871, P50=0.6899327566500407, P51=0.6475513430340497, P52=8.076905755620294e-05, P53=4.1007237465671756e-05, P54=5.22665408233224e-05, P55=1.312882913156065e-05, P56=9.41756342746742e-06, P57=3.146525455003444e-05, P58=3.906075157624537e-05, P59=6.307347737744752e-05, P60=0.00015560001234962394, P61=0.0005460524185734734, P62=0.0027717281936509262, P63=0.008633895396819389, P64=0.02120098013922629, P65=0.03036094180422385, P66=0.023321189483650508, P67=0.012725662215679745, P68=0.002016399667633187, P69=0.0006579218150037104, P70=0.0005481718737371261, P71=0.0004649581499247059, P72=0.0008317508777971974, P73=0.0009615899455622633, P74=0.0015197146409691446, P75=0.0022284429834304666, P76=0.002490943686271671, P77=0.006999658257540225, P78=0.02028880446676023, P79=0.014420780403848008, P80=0.003152618185331865, P81=0.0043929005542402595, P82=0.00376974282646059, P83=0.0011567773016394529, P84=0.000735186865716902, P85=0.0007412926662270134, P86=0.00035029337122591183, P87=0.0005978341183577991, P88=0.000999057808701544, P89=0.0007625912899611546, P90=0.0005849769161767225, P91=0.0004969265580130167, P92=0.00036781792063296753, P93=0.0030380833795994565, P94=0.010128125412081324, P95=0.004668106979420365, P96=0.0010113721766152684, P97=0.0010943964591655566, P98=0.0005574993337576977, P99=0.00017807992366538828, P100=5.788942809388011e-05, P101=9.758597239596968e-05, P102=0.001374147476638508, P103=0.001233625001356045, P104=0.00026138260705456623, P105=6.659999080165694e-05, P106=2.8924880987802874e-05, P107=2.0840055829664165e-05, P108=8.370573694169724e-05, P109=0.003153877886186463, P110=0.002062297816333443, P111=0.0001517029839894274, P112=4.0292958200179724e-05, P113=5.0573685945143634e-05, P114=0.0005183364063202624, P115=0.0018697619896849815, P116=0.0003346266455467911, P117=3.9163848390046486e-05, P118=3.9992074082453265e-05, P119=0.00039882714859064243, P120=0.0006011707868949629, P121=0.00021621168756298122, P122=0.00012175337780607948, P123=7.820366394363061e-05, P124=0.00011538293061207465, P125=0.0001329068045335139, P126=2.1092877075103738e-05, P127=2.1951835370172224e-05, P128=1.4917059583007739e-05, P129=9.217430965683349e-06, P130=6.794938489172434e-06, P131=5.860963304493472e-06, P132=4.3668399052804455e-06, P133=3.0312489829397e-06, P134=5.350986493729518e-06, P135=3.586743013962119e-06, P136=5.242654482981732e-06, P137=8.088083521454324e-06, P138=1.1411800190988618e-05, P139=2.889340925727015e-05, P140=4.3958889522375105e-05, P141=5.873519942724078e-05, P142=4.578478501461325e-05, P143=4.463759869433057e-05, P144=1.9526527081137934e-05, P145=3.348706944751034e-05, P146=4.622376896070159e-05, P147=5.572738482808763e-05, P148=3.7222253042291714e-05, P149=4.1983035653108484e-05, P150=3.073743618935942e-05, P151=1.7395129360515344e-05, P152=2.6297613655367944e-05, P153=2.4266812534732962e-05, P154=2.4789671505281782e-05, P155=2.3403824870258696e-05, P156=3.36383984829297e-05, P157=2.927113871371388e-05, P158=2.318367406442415e-05, P159=2.9106496441399273e-05, P160=3.881383152286659e-05, P161=5.321548836340448e-05, P162=6.110972935915192e-05, P163=3.5845652234618176e-05, P164=4.130338611490265e-05, P165=3.712646395147267e-05, P166=2.459209422880077e-05, P167=3.822373426758871e-05, P168=8.782430589532782e-05, P169=0.0001141980645905022, P170=7.569320946579811e-05, P171=5.933541690297351e-05, P172=1.8430011580818103e-05, P173=7.662329847626974e-06, P174=1.0699114102682428e-05, P175=8.33321515508759e-06, P176=7.890336554125976e-06, P177=5.1534214934434315e-06, P178=3.403733468898944e-06, P179=3.987793419336382e-07),Row( P0=-626.0147719728201, P1=52.24580668927609, P2=-4.58559693431049, P3=1.0690275459135032, P4=-11.149141398910531, P5=-5.8128074520562985, P6=-1.4295185636904064, P7=3.5522967224795035, P8=8.38112107560342, P9=-6.149982573923804, P10=-1.1908512844406882, P11=3.2200851422274566, P12=1.3064111532331482, P13=4.983929674659737, P14=2.5698077213103883, P15=4.77160919878243, P16=5.4466165617874704, P17=10.96997500802239, P18=11.313997929237878, P19=8.364951870938645, P20=7.291578589145946, P21=8.83736264220277, P22=3.897164449892453, P23=4.105937313921924, P24=1.831653093269201, P25=2.2503344125511564, P26=-0.5781798803374602, P27=1.3435884494000172, P28=-0.09492161955277645, P29=0.454152935454297, P30=0.47738225242793936, P31=1.2726271357594394, P32=0.04845988657573607, P33=0.8601132621548397, P34=-1.863262226237232, P35=0.9511660336189649, P36=-0.08421884673152666, P37=1.7212978407345128, P38=-0.3469359662135556, P39=-0.7273415854858392, P40=0.645369522970496, P41=0.6072796005857719, P42=0.5717958585980287, P43=0.5756097779125846, P44=0.6068522880022474, P45=0.6478784187420676, P46=0.6847068392820542, P47=0.6707076118738847, P48=0.6670017141712471, P49=0.6950054315116487, P50=0.6826214667035374, P51=0.655871296439244, P52=0.004892887396238509, P53=0.0001915732358307611, P54=5.461612741911072e-05, P55=1.1744031720432683e-05, P56=1.2373397447244839e-05, P57=1.690352312043029e-05, P58=1.4369209503465226e-05, P59=1.5900618881674922e-05, P60=0.000101837837486421, P61=0.0009251482105490662, P62=0.01046539658976482, P63=0.02346503005791186, P64=0.024608084837445518, P65=0.051834264665519394, P66=0.20219861938205874, P67=0.22914277711914835, P68=0.07109394110268905, P69=0.025635124219170715, P70=0.003805733876475114, P71=0.0006068522844063754, P72=0.0013772622399346962, P73=0.003260621291169201, P74=0.016423197222837867, P75=0.020354648153796295, P76=0.0278942453405398, P77=0.046403078199662315, P78=0.04946595626339216, P79=0.030442809329869643, P80=0.015636331837025893, P81=0.011562732567551897, P82=0.010349482455459035, P83=0.005436493335997128, P84=0.009879718004124838, P85=0.04472254303259588, P86=0.05215476797745832, P87=0.016670049424325224, P88=0.026751426658453693, P89=0.024221662168457916, P90=0.0187069339345439, P91=0.012221422050998703, P92=0.020766844144181295, P93=0.01430836835519553, P94=0.011228833828886115, P95=0.008500890728217953, P96=0.0013106893475269116, P97=0.0005283357595688389, P98=0.005484182504506532, P99=0.009083781100328639, P100=0.0024462524614944493, P101=0.002882463273344389, P102=0.0010095998287188972, P103=4.614040351272259e-05, P104=4.050370605729599e-05, P105=0.0002541225609702343, P106=0.0003581040654371126, P107=0.00022254683655952663, P108=0.0003184755276391537, P109=0.0003356605120537812, P110=0.0004762343372223059, P111=0.0002968338222516177, P112=0.0004920589479604096, P113=0.0007752497667081824, P114=0.0006390785561209044, P115=0.0007943738040521182, P116=0.0005326957639246253, P117=0.0012393440843902702, P118=0.001238688241250521, P119=0.0004351415837418998, P120=0.0006756617676743922, P121=0.0006518599169694578, P122=0.0005108539615768152, P123=0.0008439710795972333, P124=0.0023224242338884733, P125=0.0006707945760034624, P126=0.00093151444227614, P127=0.001596001554535368, P128=0.00018343001042205536, P129=0.00011598780339003969, P130=0.00011440705385022767, P131=8.635319253927352e-05, P132=0.00010104351873534427, P133=4.942894673449444e-05, P134=3.2663822724345325e-05, P135=4.976088126332941e-05, P136=5.247080955778954e-05, P137=9.482365584631498e-05, P138=9.217627827298741e-05, P139=8.754994136953355e-05, P140=6.314504617074257e-05, P141=6.598957907025734e-05, P142=5.758575039918492e-05, P143=0.00010251330519843133, P144=8.050361337423847e-05, P145=0.00010220107734952096, P146=0.0001268876154953802, P147=0.00011701953229136661, P148=0.000144970025111476, P149=0.00019191242667859851, P150=0.00017989667867211726, P151=0.00014945884822380664, P152=0.0001053160369184733, P153=0.00010414286612631454, P154=5.7712683409850716e-05, P155=3.755556474289283e-05, P156=2.6194762970180402e-05, P157=2.1752863667083005e-05, P158=2.576446474089455e-05, P159=3.9130781393542e-05, P160=6.147287047590347e-05, P161=5.963599972234409e-05, P162=3.6454486595469694e-05, P163=5.501970596200561e-05, P164=6.02328247716948e-05, P165=5.889958291515026e-05, P166=6.861705313295243e-05, P167=5.508028512952947e-05, P168=7.187368980276491e-05, P169=8.140532225161821e-05, P170=3.494560694378839e-05, P171=3.375681449129576e-05, P172=2.1011747031473094e-05, P173=1.948323010537646e-05, P174=1.8368467644551254e-05, P175=1.6456389219962053e-05, P176=2.126788973910479e-05, P177=2.8097249193258458e-05, P178=1.441777174758127e-05, P179=1.3333076186049777e-06),Row( P0=-556.5435391820035, P1=35.46897354086781, P2=-28.1678371925869, P3=7.431493077141729, P4=-16.37254346111372, P5=-16.12836146182272, P6=-3.273337667056611, P7=-8.289209398873512, P8=0.3835516839244081, P9=-5.584699855791534, P10=3.530500244637464, P11=6.283951334461476, P12=6.700151274494263, P13=16.817108999313394, P14=17.259897264879662, P15=18.679898152235932, P16=11.688978210640043, P17=10.2696390063364, P18=7.908560477080622, P19=3.9061010839602264, P20=4.003542985840701, P21=8.359647074594651, P22=3.1988428352806015, P23=2.3227860478491555, P24=-3.123829027660315, P25=-2.1227691507475637, P26=1.8949319554817747, P27=4.586344103829604, P28=-0.2711706902990936, P29=1.8621727973834628, P30=1.0936120919668375, P31=3.4809088315758356, P32=1.5574291648426268, P33=-0.059291742619740205, P34=-1.7890013067239148, P35=2.780449086592704, P36=1.7617306034151965, P37=1.9377948364300897, P38=-1.9303197344730658, P39=-0.551818474169127, P40=0.6176299783110936, P41=0.6369570751759861, P42=0.664555250650452, P43=0.5794827758232152, P44=0.5290273981604633, P45=0.5605844729123808, P46=0.5988044996718505, P47=0.5746836424672179, P48=0.5911026476635376, P49=0.619519408999494, P50=0.6177324338844672, P51=0.6054824824589159, P52=0.002951850443093719, P53=0.0001419720364127152, P54=6.093448975531669e-05, P55=1.2169849739374613e-05, P56=4.3230254867623105e-06, P57=4.712598898266034e-06, P58=5.483035956887067e-06, P59=4.419468505436473e-06, P60=6.1437394106534255e-06, P61=6.610033866855873e-05, P62=0.0025785192356106365, P63=0.004793181989055149, P64=0.0045614593865787876, P65=0.027784400413714525, P66=0.10200485281146124, P67=0.13730040116496112, P68=0.2227776202254071, P69=0.6507496597788792, P70=0.9270964440586463, P71=0.36405456597385016, P72=0.11493344492580179, P73=0.06580216154975345, P74=0.030046324844312366, P75=0.013160908802939186, P76=0.004220523961634722, P77=0.0031996552303411112, P78=0.0028621480971683182, P79=0.004048316196235998, P80=0.010171307074981182, P81=0.02888242548903117, P82=0.0661625746059852, P83=0.07544247038265026, P84=0.06818468763344523, P85=0.06364651232127803, P86=0.06104234380033192, P87=0.08664081983184102, P88=0.2200765781394141, P89=0.14744243148297165, P90=0.04843917886581279, P91=0.022528240040668307, P92=0.029042552227041656, P93=0.028985006027804065, P94=0.04368283859081448, P95=0.028099679720871636, P96=0.0084656479975753, P97=0.008573650917493365, P98=0.08349507958401241, P99=0.11050025835435409, P100=0.3588570881639613, P101=0.19984997149497546, P102=0.037059785575886545, P103=0.010197498400861338, P104=0.005472231453591654, P105=0.0025227239318201425, P106=0.0017426839313898137, P107=0.007704378436517949, P108=0.07747723708701822, P109=0.1266100474643703, P110=0.07173234042111938, P111=0.037128356204562125, P112=0.0396493283774418, P113=0.010342652483357787, P114=0.008986129568964142, P115=0.14170062925038973, P116=0.23697781841657387, P117=0.0460559308279178, P118=0.009997455900646872, P119=0.00867363431464773, P120=0.007844686274880833, P121=0.014740256940233771, P122=0.007921201796912471, P123=0.004251092813814072, P124=0.007398010827796466, P125=0.009008461690251214, P126=0.00938909880216989, P127=0.020414341936578054, P128=0.0204291503799026, P129=0.012775823069282497, P130=0.0033363537533210716, P131=0.011438653774540421, P132=0.011918307856304641, P133=0.0020439009184329594, P134=0.006276761227883634, P135=0.009517046810461015, P136=0.0029190189206162497, P137=0.002851263405424752, P138=0.010343506000498909, P139=0.007217997581631469, P140=0.00576212831076512, P141=0.004393700232977614, P142=0.0009460642509300563, P143=0.0012529427323239321, P144=0.0011640227060890674, P145=0.0010415252742444695, P146=0.0011112250650742652, P147=0.001280479274440828, P148=0.0022038258520561747, P149=0.0012394853409107927, P150=0.0011225259036754358, P151=0.0007342474132476359, P152=0.0005066747267884835, P153=0.0002665307107975872, P154=0.0002535991454428583, P155=0.0002882324283396108, P156=0.00020650460578254353, P157=0.00012939047678930352, P158=0.00013950546095892357, P159=0.0001845257200893643, P160=0.0002340458951603225, P161=0.0002483420834978935, P162=0.00020100612996850242, P163=0.00011124785563335703, P164=0.0001474426458680724, P165=0.0001695303547004752, P166=0.00014192792352472262, P167=0.00014627359150497978, P168=0.0002122643306006164, P169=0.0004050605992190301, P170=0.0002367166924785745, P171=0.00043455720384351853, P172=0.00016423112783805135, P173=0.0001236578379140459, P174=0.0001334757281717392, P175=0.0002534329525033804, P176=0.0006055589777493638, P177=0.0011223191623553023, P178=0.0005404307875629894, P179=6.722190223856202e-05),Row( P0=-673.9960710793105, P1=60.413022174659154, P2=-4.29289281281153, P3=11.872504963718097, P4=-8.065318783253842, P5=-8.043316238985536, P6=-0.12710818415950706, P7=-2.402144159539937, P8=2.484344628177476, P9=-4.734594161594511, P10=1.6070099005453675, P11=0.4901791695038126, P12=1.2089408012737588, P13=1.6034572632714512, P14=3.9990326558465425, P15=6.411688955620661, P16=4.537591286665491, P17=9.765396557516532, P18=11.929232310913791, P19=9.129078097959443, P20=5.0636207493371, P21=7.607972861849357, P22=1.9244503360975656, P23=5.9883536984546355, P24=1.9209761611455076, P25=1.6260973052121748, P26=-0.9029627184868328, P27=-1.5403678685047995, P28=-3.262502643695483, P29=2.0179899670545867, P30=-0.48558184283639944, P31=0.3914513219691276, P32=-1.5906652603064646, P33=1.7006913267437436, P34=0.7094996585404242, P35=2.3096570964902963, P36=-1.8784411595783586, P37=0.8164385398066021, P38=-0.10760069291948222, P39=2.1137676590139023, P40=0.6520436845869031, P41=0.5731184237301176, P42=0.5673062066202521, P43=0.5855718157678557, P44=0.6423514991832877, P45=0.6861972004057604, P46=0.7329220997889, P47=0.6852370739210301, P48=0.6857538622857705, P49=0.7056442916110308, P50=0.6699109167508078, P51=0.7087663164124378, P52=6.511174079328662e-05, P53=3.673438706933183e-05, P54=2.4629428310509026e-05, P55=4.404428549890007e-06, P56=1.0516443731921641e-05, P57=8.515796492315296e-05, P58=7.396475681097145e-05, P59=5.585514818676963e-05, P60=0.00010770309680880457, P61=0.00048758622741134396, P62=0.005081755505280073, P63=0.009331171424770026, P64=0.025182317826782635, P65=0.05961240378922357, P66=0.07114035770446035, P67=0.11869210117892626, P68=0.04180422449235558, P69=0.005178535071176361, P70=0.002452197217626089, P71=0.0011084189532746943, P72=0.001218066273680482, P73=0.0017367501788761877, P74=0.0032071224126905026, P75=0.0036878287346412157, P76=0.003646855105241334, P77=0.014510256622985492, P78=0.052500553770241896, P79=0.03612750425058252, P80=0.014344354191745505, P81=0.007310070974550865, P82=0.012196152403725315, P83=0.02467272404207619, P84=0.019957525212535163, P85=0.010575447796691692, P86=0.003003763674914644, P87=0.002466274971344724, P88=0.0029721687315123887, P89=0.0017932449731072357, P90=0.0019882313475354887, P91=0.0029565029727174185, P92=0.0029282977548952486, P93=0.0016368903487683328, P94=0.005816819181785993, P95=0.057593014979904815, P96=0.027787844185137925, P97=0.011831400510965595, P98=0.0022569932240859656, P99=0.0003833532166510064, P100=0.0002822920611418515, P101=0.0003239552467950333, P102=0.00023233779452209113, P103=0.006198668188725292, P104=0.0319060543409192, P105=0.004402680453655065, P106=0.00038843524604444216, P107=0.00012363881101072843, P108=0.00012503529066241155, P109=0.0005708922297680265, P110=0.0016105177884625681, P111=0.003705317649539071, P112=0.0013055907400977725, P113=0.0003614400676474053, P114=0.0003871443711740505, P115=0.0003038864482763292, P116=0.019811519778551553, P117=0.012521245813096606, P118=0.0016410061155494563, P119=0.0003346190934399283, P120=0.0003694373375720342, P121=0.004617651534760123, P122=0.0037585658442716146, P123=0.0008996566125278005, P124=0.00037012357451486844, P125=0.0011905934929020022, P126=0.00043010565271048464, P127=0.00025524954493504146, P128=0.0003992928954576869, P129=0.00029155196252860893, P130=0.0002471822616597462, P131=0.0001570924774066644, P132=0.00010004760603847246, P133=0.00010776282074780772, P134=5.007772764555499e-05, P135=0.0001816768127905277, P136=0.0001517564649024114, P137=0.00028876396399431394, P138=0.0006269054467818706, P139=0.00031748816661630484, P140=0.0011692944877955374, P141=0.0003873012847561791, P142=0.0001344452692452217, P143=0.00011217136352483006, P144=5.297770810435844e-05, P145=8.025738526166812e-05, P146=6.244857632010334e-05, P147=0.00010900665268754872, P148=0.00014882151631294023, P149=0.00012679017735344177, P150=0.0001082468053882387, P151=5.6444286367006866e-05, P152=5.3585267288539354e-05, P153=3.174890876411042e-05, P154=1.53292749096643e-05, P155=1.3679352626970083e-05, P156=1.59283384597534e-05, P157=2.7141702194987225e-05, P158=2.6313805842498867e-05, P159=3.324210870055297e-05, P160=2.4229716098083064e-05, P161=5.087593267556269e-05, P162=5.6756965157972874e-05, P163=6.489783344411721e-05, P164=2.632938345153221e-05, P165=2.397189736544518e-05, P166=3.783942742346627e-05, P167=4.474203559129841e-05, P168=6.111135749479033e-05, P169=7.353678627492494e-05, P170=6.261586916231576e-05, P171=6.891303288793616e-05, P172=7.53168131154541e-05, P173=7.37164483388045e-05, P174=4.930290688160767e-05, P175=4.5874689435582424e-05, P176=4.981081320738349e-05, P177=9.35919306993741e-05, P178=4.3727651041744166e-05, P179=3.6325222019784026e-06),Row( P0=-735.4360621699892, P1=75.50989274287963, P2=9.119369066846094, P3=12.650396580938832, P4=-5.813580660983666, P5=-1.6621220455535242, P6=-4.273008634276047, P7=-1.1455630315210907, P8=3.809886453821587, P9=-0.8430458658627812, P10=2.7557846227247045, P11=4.128656896227252, P12=3.1824701591753874, P13=-1.8431078128636402, P14=5.170896897438569, P15=2.504384835178687, P16=4.5620445143537935, P17=10.454125967104353, P18=12.321842094780454, P19=13.1379941047591, P20=8.696092434058674, P21=9.375942146766148, P22=4.467961801894242, P23=2.7992242349901484, P24=-2.6822593257622773, P25=-1.2471385640454713, P26=-1.2811831879299935, P27=0.5246342553262584, P28=-3.670628362540849, P29=0.19722054143107592, P30=-4.045789517792162, P31=-0.4800666310124597, P32=-1.152063394471727, P33=0.785505072603112, P34=-0.04934898271645548, P35=0.025058669393205203, P36=-0.5768857997925578, P37=3.063183041452222, P38=2.093414667902549, P39=2.471102886102921, P40=0.6297528454013085, P41=0.5817649025844139, P42=0.5669633550010843, P43=0.5576337035205359, P44=0.5872283762584782, P45=0.6195825003730644, P46=0.6608258989793259, P47=0.7065254890871114, P48=0.8005645055378428, P49=0.8100982198002006, P50=0.7526351960064811, P51=0.6466716113753039, P52=5.372976775985708e-05, P53=4.1044559785305826e-05, P54=3.4584333820008744e-05, P55=3.897355511684358e-06, P56=2.838663620431391e-06, P57=1.0128732273804928e-05, P58=1.0035634383268103e-05, P59=5.121272117896277e-06, P60=1.360254087391485e-05, P61=5.883320182484438e-05, P62=0.0004967507889283502, P63=0.0037292943079511743, P64=0.012081626492294363, P65=0.023851457991280616, P66=0.022077172151067224, P67=0.013690284849005899, P68=0.004253736875116392, P69=0.0008759345399930867, P70=0.0003862264126759534, P71=0.00014012674018934685, P72=0.00012338670386803516, P73=0.00018539086925217497, P74=0.0007136047140262067, P75=0.0010674018185691156, P76=0.002467651102453621, P77=0.01047185210083839, P78=0.008856560711943096, P79=0.006736195879378138, P80=0.006143964978834027, P81=0.0014097781186834643, P82=0.0007741459261677343, P83=0.0007774264062604553, P84=0.0006610185995439105, P85=0.0003817924851657243, P86=0.00016441868866069598, P87=0.00023945849147130133, P88=0.0005055479905653073, P89=0.0009695642709168638, P90=0.0008714489195501555, P91=0.0008141759583150355, P92=0.0024123513707713863, P93=0.0008652912382564528, P94=0.0004903349828774026, P95=0.0005234028264111017, P96=0.00020529190873489546, P97=0.0006143881692240668, P98=0.002131343863407425, P99=0.0004232388063051108, P100=0.0004785325213628351, P101=0.0018639588461762092, P102=0.0005889722589336172, P103=5.9657411429017776e-05, P104=8.810704923236402e-05, P105=0.0003723102819481268, P106=0.00010514749894898976, P107=0.0009124757627640132, P108=0.002104560729445334, P109=0.00026607399219129237, P110=0.0002862246181349508, P111=0.00014527886938012303, P112=8.553274763940868e-05, P113=0.0005279462942819675, P114=0.0002970984494008961, P115=7.57792578675465e-05, P116=3.311196674706229e-05, P117=6.353896719721423e-05, P118=0.00046448397036907596, P119=0.0002343727937523191, P120=8.418909921736134e-05, P121=0.00014612651791976586, P122=0.00015372797341620497, P123=9.467321263079431e-05, P124=2.648169884143214e-05, P125=1.87827723875607e-05, P126=6.520090546579692e-05, P127=9.34823452653648e-06, P128=5.949659436028233e-06, P129=1.0771765287589155e-05, P130=6.3096651360300575e-06, P131=4.253897874605033e-06, P132=4.601563368055297e-06, P133=2.7138967197711003e-06, P134=4.156803906704418e-06, P135=4.231978681839728e-06, P136=4.7055931612150426e-06, P137=7.514909135415699e-06, P138=7.754521178178384e-06, P139=1.456108352592381e-05, P140=2.0077626913470542e-05, P141=2.7133573107833576e-05, P142=1.846785602478238e-05, P143=1.1515960135142428e-05, P144=9.26014886099866e-06, P145=1.3981815157457241e-05, P146=8.977234161400352e-06, P147=1.0154032759178746e-05, P148=1.1401568259888589e-05, P149=1.1168356771694575e-05, P150=7.478054411722148e-06, P151=4.56506561899382e-06, P152=3.721915830834681e-06, P153=3.158112620563328e-06, P154=6.870815755791758e-06, P155=9.976188318055835e-06, P156=9.136012128907137e-06, P157=1.4419937503872406e-05, P158=1.6633171785375165e-05, P159=8.079561807846831e-06, P160=7.89107521238898e-06, P161=1.4438851711405962e-05, P162=8.837916254269532e-06, P163=9.61030843315383e-06, P164=8.526033639318569e-06, P165=5.986056787502557e-06, P166=8.576026405970842e-06, P167=6.839920486640135e-06, P168=3.6955781093154896e-06, P169=2.3572278573957162e-06, P170=1.3650933405557352e-06, P171=6.178336900692668e-07, P172=1.0010830236154897e-06, P173=1.7832416898936128e-06, P174=1.890404214506988e-06, P175=1.2899431437050142e-06, P176=1.1967326909956278e-06, P177=1.5465038653499024e-06, P178=8.911212588706054e-07, P179=1.3895262777993223e-07),Row( P0=-704.3396600483328, P1=61.246508678274914, P2=8.319039355944241, P3=4.3104738922713315, P4=-3.3280788458509383, P5=-4.826246003923426, P6=1.1315852910570787, P7=1.4545359418214932, P8=6.825396458350696, P9=-1.1788960728647822, P10=-1.7810387651082766, P11=2.628108927888377, P12=-0.598976796782604, P13=1.5690221112862166, P14=-0.2566724894882284, P15=4.93306452227651, P16=2.4140589794527227, P17=9.195712467418662, P18=10.35104354320255, P19=11.86501983698762, P20=10.555649095138996, P21=9.978160411008039, P22=4.263986889003082, P23=3.4193431043256117, P24=0.930246062232809, P25=0.7732597005770563, P26=0.6706567021751214, P27=2.479887249178994, P28=-2.055487485848945, P29=-1.3820014760129808, P30=-2.85224031064079, P31=-0.5603977521943257, P32=-0.16579452933372235, P33=-0.9724453248726196, P34=-0.8260073857075181, P35=-1.604203364822569, P36=-1.5677951647917046, P37=0.8817475502468075, P38=-0.378027063095428, P39=0.8602094308402435, P40=0.6204970505602457, P41=0.5966752471795755, P42=0.6168530383815308, P43=0.6382738864808676, P44=0.6640158918872399, P45=0.6577934491011822, P46=0.687734473669765, P47=0.6860073377152742, P48=0.7051264029391306, P49=0.7061347987783717, P50=0.674576688790242, P51=0.6321512914053046, P52=3.926885596925764e-05, P53=3.365070594092918e-05, P54=3.9031131838779564e-05, P55=4.55836393341456e-06, P56=2.716571075866966e-06, P57=4.646430606784375e-06, P58=1.978331043362549e-06, P59=5.968875851709741e-06, P60=0.00019359741576063786, P61=0.0014367945035864945, P62=0.0070750639389858625, P63=0.019964704897911707, P64=0.024912804606150474, P65=0.07632122027314582, P66=0.07058127742230169, P67=0.029122927412685823, P68=0.005256583835305508, P69=0.000648296257247846, P70=0.0004974467554446477, P71=0.0007023687125626124, P72=0.001320236000458945, P73=0.0012857318387443737, P74=0.005698088604366078, P75=0.01334600568239191, P76=0.01691888159474124, P77=0.019701610435904668, P78=0.020687251697465908, P79=0.0236095355101487, P80=0.028259532142497476, P81=0.014548335266250243, P82=0.01183334173871267, P83=0.00593281024119215, P84=0.0027884025509335455, P85=0.0034676210299826905, P86=0.003135985439019933, P87=0.00661437067488418, P88=0.013052663490352065, P89=0.007976046906688558, P90=0.004213456217410914, P91=0.0033254009547484657, P92=0.002592541787605443, P93=0.0008627059918161534, P94=0.0006008036283621917, P95=0.000559373543453844, P96=0.0003293425666617587, P97=0.0018067382008172368, P98=0.0040453583746080995, P99=0.002251231681618097, P100=0.0007186627416225601, P101=0.0004769514968937511, P102=0.00032032944200122143, P103=5.197489123204735e-05, P104=0.0001944655503704285, P105=0.0006643959477523987, P106=0.0003291849480882765, P107=0.00010975918260662163, P108=0.00020425306613519747, P109=0.0001393848739582765, P110=0.0001772844919637719, P111=0.00029336618248633256, P112=0.00023335011538953482, P113=4.18628867613776e-05, P114=6.381020792403454e-05, P115=0.0004467979227075268, P116=0.00029529900170223417, P117=0.000133140892428625, P118=0.00011310934699187557, P119=0.0001758754111982719, P120=0.00012110301368581544, P121=0.00013241945283083375, P122=9.71183296472061e-05, P123=0.00010325633844276995, P124=0.00011355654497652568, P125=0.0001558932066817301, P126=9.366804030835753e-05, P127=4.619743111667269e-05, P128=6.561695609565323e-05, P129=6.521059201624883e-05, P130=5.639940343916483e-05, P131=3.3555741043061644e-05, P132=1.3612941107751334e-05, P133=1.1444061256782738e-05, P134=1.4314885665937757e-05, P135=2.3854587740878504e-05, P136=1.4458572290224253e-05, P137=3.0586376155967656e-05, P138=4.214144917962917e-05, P139=2.897050186563417e-05, P140=4.027974373085259e-05, P141=2.0720583062014348e-05, P142=3.4237625775265005e-05, P143=1.7422151187723402e-05, P144=3.091014868702965e-05, P145=2.6050006252237107e-05, P146=4.606544919516008e-05, P147=7.188938316611169e-05, P148=5.721602184107609e-05, P149=5.9173887448803616e-05, P150=7.603807281954279e-05, P151=3.7476310982916385e-05, P152=4.495126693538655e-05, P153=7.408609061020923e-05, P154=4.9331561808346934e-05, P155=4.087365831933717e-05, P156=5.2762511051046995e-05, P157=4.5993630305330293e-05, P158=2.8820872145546024e-05, P159=2.4792278070057746e-05, P160=2.026020458583935e-05, P161=2.206526447108555e-05, P162=3.4391274181723214e-05, P163=4.550568028402456e-05, P164=8.450324522538016e-05, P165=0.0001282989914295373, P166=0.00012172065437820553, P167=0.0002501992224267906, P168=0.00045712292818078545, P169=0.0004627130461665586, P170=0.0005195892643339483, P171=0.0005713506799614126, P172=0.0006055344621417683, P173=0.00042174142159995666, P174=0.00046407518826559625, P175=0.00032991459607886335, P176=0.0002202802795979517, P177=0.00012077557892175727, P178=4.220151103112688e-05, P179=2.711329674215152e-06),Row( P0=-736.6187921407653, P1=76.81469574232818, P2=9.81046059642895, P3=14.786215243306048, P4=-4.339231163042634, P5=-4.162678567673942, P6=-2.529892516649402, P7=-1.3614349506690457, P8=7.508826030288524, P9=-1.0600525742770333, P10=4.007230216052787, P11=3.7514064003906764, P12=4.141689045009698, P13=-1.358259654392431, P14=5.064972433908597, P15=2.060934461652805, P16=6.008526945420553, P17=12.24228972970513, P18=13.348303537505274, P19=14.443272293625323, P20=9.908477188371988, P21=8.272563536297264, P22=0.9654811971258069, P23=-0.9707540738433938, P24=-4.207331964328094, P25=-0.02078851081290149, P26=-0.11785026348982257, P27=0.7879818330788148, P28=-5.513535218826997, P29=-1.1441745468476312, P30=-4.2438540286444715, P31=-0.6808745084482689, P32=-1.9003339731151767, P33=-0.712712332010455, P34=-1.7312156089855693, P35=1.399130722313257, P36=-0.27676635722702486, P37=3.2925563432082066, P38=1.1985579855033226, P39=2.298194668686664, P40=0.5965617392933567, P41=0.5511661670956759, P42=0.5693244203717283, P43=0.5961352493716996, P44=0.57138668383878, P45=0.5829681202495381, P46=0.629075169637022, P47=0.6330883289742026, P48=0.7205375406193194, P49=0.806056509282924, P50=0.8104960808167972, P51=0.6961628166152889, P52=6.517803470241067e-05, P53=3.331245165289464e-05, P54=3.7620091928994814e-05, P55=8.781358437920037e-06, P56=1.0216566604923089e-05, P57=3.3987982786510334e-05, P58=1.4880882257042093e-05, P59=2.418340643716474e-06, P60=2.149971004421911e-06, P61=6.113731831245463e-06, P62=0.0001555341511995685, P63=0.0016866116724485547, P64=0.011456520170150678, P65=0.03662246705489549, P66=0.026352507013199303, P67=0.00798406668623854, P68=0.0012298076002122743, P69=0.0005057744682902504, P70=0.0003133218561021173, P71=0.0001295807704563106, P72=0.0001075414180396671, P73=9.3273700494451e-05, P74=0.0002370628763326989, P75=0.0005122879302290594, P76=0.0015909926965714614, P77=0.004238795936478788, P78=0.011280787633601809, P79=0.015682115685301555, P80=0.008496205973306454, P81=0.003984921010646114, P82=0.001656520432327825, P83=0.0004175674320561462, P84=0.00041892115662556, P85=0.00035133861216037364, P86=0.00014452012285966017, P87=0.0007987975389320241, P88=0.002518504636782375, P89=0.0015540070357690184, P90=0.0008942752909216728, P91=0.0007165983082113085, P92=0.0007043056932379653, P93=0.001973140345813931, P94=0.003330205423483426, P95=0.0005327822493286311, P96=7.602315851866509e-05, P97=0.0012169458718193059, P98=0.002019353649975134, P99=0.0003018999964360496, P100=7.987568619926152e-05, P101=0.00010638879524528478, P102=0.0016041852481558756, P103=0.0009604004559690769, P104=0.0002827586529392747, P105=0.00019991684527572579, P106=4.266375672668066e-05, P107=1.2913841562397885e-05, P108=4.2983052163504085e-05, P109=0.0007412750552890886, P110=0.00048279391935050005, P111=5.2000554468900545e-05, P112=4.448831122092467e-05, P113=3.579419726980162e-05, P114=0.00022138622270355106, P115=0.0008979975141577358, P116=0.00011002864723971407, P117=2.158004436184655e-05, P118=5.766510277068765e-05, P119=0.00035136328416675766, P120=0.0002527665632763123, P121=4.701157564029072e-05, P122=5.605945998284523e-05, P123=6.961917852021833e-05, P124=4.685393929614231e-05, P125=8.926695618880258e-05, P126=0.0001907702068950365, P127=1.832939560122241e-05, P128=8.657163059389401e-06, P129=1.2585710764104062e-05, P130=8.965952721042799e-06, P131=1.2661741666207427e-05, P132=1.3658548778095446e-05, P133=4.416635958264634e-06, P134=6.690979566434825e-06, P135=4.117026787607112e-06, P136=1.5420362577567438e-05, P137=1.18629447068664e-05, P138=1.1460780246779179e-05, P139=2.2379662229436236e-05, P140=3.86604681691879e-05, P141=3.571555134862314e-05, P142=2.1963731912459604e-05, P143=1.9141572876392332e-05, P144=1.3569355293677405e-05, P145=1.745397311749953e-05, P146=1.3600282430065715e-05, P147=2.154211762236532e-05, P148=1.1444801035758448e-05, P149=1.0057213709325583e-05, P150=6.197566831829232e-06, P151=9.360661763996939e-06, P152=4.1789843574389286e-05, P153=6.089930265291446e-05, P154=2.3040319432531597e-05, P155=1.5969043524998956e-05, P156=1.3014482498929548e-05, P157=1.384571900661009e-05, P158=1.7083542073582936e-05, P159=1.865654501543725e-05, P160=2.395503183311791e-05, P161=2.2774335808728585e-05, P162=1.6816240480313766e-05, P163=1.754656978890431e-05, P164=1.4614132612190964e-05, P165=1.27592174888115e-05, P166=1.2088216089315814e-05, P167=1.0606009335331322e-05, P168=5.897685052185796e-06, P169=1.2372570227807623e-05, P170=2.0922466874663418e-05, P171=7.206795581371531e-06, P172=1.78811330406827e-06, P173=3.3362528842814452e-06, P174=1.4113436800411808e-05, P175=1.3345356961887697e-05, P176=3.167682325303713e-06, P177=3.1718150988990427e-06, P178=3.6026607304082613e-06, P179=4.896269424021194e-07),Row( P0=-673.24758235318, P1=59.70820191924604, P2=3.815518918754791, P3=0.49038032666769255, P4=-10.792051350952347, P5=-9.698757345953938, P6=-2.83033234247561, P7=-0.8363913040443809, P8=1.9307338696162029, P9=-2.845789732921783, P10=0.8188566741677517, P11=0.38546311637657577, P12=-1.0294139519517376, P13=4.94656797100789, P14=-3.030637201174841, P15=3.6222289921155113, P16=2.906725764497358, P17=9.235724850861631, P18=12.85231047817908, P19=12.002369829049695, P20=9.727139073862528, P21=10.15664361340942, P22=4.132234259104454, P23=3.7807353243923134, P24=1.6491932235242805, P25=3.4303832446457245, P26=1.3217785754012874, P27=0.3556802631745489, P28=-1.8370241095593218, P29=-2.086668564762789, P30=-0.7089847114487405, P31=-0.41158370014248513, P32=-0.43644409283783475, P33=-0.6241844731903317, P34=-3.8772741357119305, P35=-0.8248246219375194, P36=-0.2117455535889716, P37=2.5100410847444636, P38=-1.6628470095313082, P39=-1.0853098445634002, P40=0.6265404732841797, P41=0.6121713344195895, P42=0.6397235902423027, P43=0.624008736700121, P44=0.6544962910128435, P45=0.7000567125347881, P46=0.6982659504399401, P47=0.681984810523266, P48=0.6548111929363929, P49=0.7047631532050804, P50=0.7078312789948762, P51=0.6354346599970911, P52=3.9672255661990895e-05, P53=2.0348414128449502e-05, P54=1.7851110455208357e-05, P55=5.309098975984345e-06, P56=2.5353511543893895e-06, P57=1.939661288028527e-06, P58=1.55255676536973e-06, P59=4.218547800789095e-06, P60=0.0001404379732758866, P61=0.0014537932289564557, P62=0.011692405716590464, P63=0.029455825015293712, P64=0.03268392235087035, P65=0.12055869370760809, P66=0.09437830812873638, P67=0.022601211394723583, P68=0.00410797711773105, P69=0.0019443027926791365, P70=0.0013291332726294842, P71=0.0006976619006062747, P72=0.002110585343278452, P73=0.004016351591000387, P74=0.0125965715936199, P75=0.03011413598725831, P76=0.029439058295454144, P77=0.019460675266786482, P78=0.022964100657319825, P79=0.04229641234722243, P80=0.05611482236948886, P81=0.040449615844170235, P82=0.019311598779791557, P83=0.009644777937434514, P84=0.012260418346714187, P85=0.009856552092983525, P86=0.006451338189820283, P87=0.01270154373831896, P88=0.012128984275413235, P89=0.0038880407453365322, P90=0.0033867074225147954, P91=0.008796712442206773, P92=0.012449885912012335, P93=0.007087310332777473, P94=0.007234762022067486, P95=0.004604625714601766, P96=0.003100623554030118, P97=0.005575374720998415, P98=0.003651110639200006, P99=0.0018065836904260482, P100=0.0009804859448728476, P101=0.0013547916995200935, P102=0.001057891837329849, P103=0.0003174892153941448, P104=0.00028755108877530637, P105=0.00035955327236487775, P106=0.0005348080658242603, P107=0.0006169230091481607, P108=0.0008452423748593861, P109=0.000692585447896676, P110=0.001341741855726694, P111=0.00028946844164503736, P112=0.000490679730080872, P113=0.0004346653080584156, P114=0.0001879223364128161, P115=0.00031918281448594884, P116=0.0001994199702494799, P117=0.0001296303106156953, P118=9.916947953928858e-05, P119=0.00023843708362363415, P120=0.0002127606726733635, P121=0.00024316413986801564, P122=0.0003576157198317615, P123=0.0004259649774361586, P124=0.0002509786996532097, P125=0.00020675401525981936, P126=9.838730677784871e-05, P127=0.00012605618150320018, P128=0.000122702452109414, P129=0.00014677187269501325, P130=0.00015274169942783964, P131=0.00011166253459330806, P132=5.072954095085704e-05, P133=3.250691854985058e-05, P134=5.4880604146099816e-05, P135=5.826190373096779e-05, P136=5.484017305258113e-05, P137=6.981935694752941e-05, P138=5.214524239370605e-05, P139=5.4381710939930997e-05, P140=2.711675608903641e-05, P141=2.3740152144103054e-05, P142=1.1097686157619359e-05, P143=3.469945366942466e-05, P144=4.487374753124395e-05, P145=6.988939638226781e-05, P146=7.53185460481399e-05, P147=0.00014091870187821762, P148=8.814035763011733e-05, P149=0.00010595285856044224, P150=0.0001366404946345808, P151=9.384016723267403e-05, P152=7.742934751115026e-05, P153=7.240795769831856e-05, P154=6.588404338658952e-05, P155=4.329206124391408e-05, P156=2.1662331924577068e-05, P157=1.862195516096884e-05, P158=1.4484548627576297e-05, P159=1.8384415100848696e-05, P160=3.182227312946201e-05, P161=5.467891435116054e-05, P162=5.0251162982959116e-05, P163=5.911245898295829e-05, P164=5.6523289565143375e-05, P165=7.138221505494917e-05, P166=7.406616370551592e-05, P167=9.007867600954965e-05, P168=7.928604345379232e-05, P169=0.00011268795379373362, P170=0.00017061493200191187, P171=0.00019395829146715072, P172=0.0002701908764430081, P173=0.0001345502939548271, P174=0.00017240695659736, P175=0.00013701682978798396, P176=8.682836669571787e-05, P177=0.000107832567338112, P178=5.402965318345176e-05, P179=6.455068918267266e-06),Row( P0=-694.3491562954176, P1=68.96549309575886, P2=4.836095254944002, P3=8.060792291345297, P4=-3.099078523641539, P5=-0.584763250846715, P6=-1.1386142613899348, P7=0.42903432558329485, P8=5.337324070491388, P9=-2.149021106128775, P10=-0.25635949029947563, P11=3.8964113037041743, P12=2.0196895719439314, P13=0.39865625842049934, P14=4.726490550718782, P15=4.831336819580139, P16=4.925128166662349, P17=8.88763610570523, P18=8.693624186487074, P19=8.180816249168105, P20=4.195018488250965, P21=5.381590172970292, P22=3.70912436347545, P23=3.548597973930827, P24=2.0392139738221493, P25=1.866767347186687, P26=-1.4222428963085079, P27=-0.01664776743098379, P28=-3.6627982332143296, P29=0.9211568637787928, P30=-1.453867557489082, P31=0.31484138272665546, P32=-1.4076380219273381, P33=0.3367199536177018, P34=-0.5882243258187488, P35=-2.1108950964930586, P36=-2.3172550639255665, P37=-0.04980399455087947, P38=-0.5579217329739006, P39=0.1887401624193484, P40=0.7192182732303302, P41=0.6204336259616597, P42=0.5988233132664177, P43=0.6314903521437196, P44=0.6875729674568781, P45=0.6762188515621685, P46=0.6718255565985425, P47=0.6552567025378591, P48=0.6893517049820944, P49=0.7020067579664546, P50=0.6956415800820324, P51=0.7085902587027417, P52=8.159208829385839e-05, P53=4.722167770903057e-05, P54=5.516374779273481e-05, P55=1.9220361129957083e-05, P56=2.1688687657922456e-05, P57=5.2647499829315984e-05, P58=1.815999436143173e-05, P59=1.3783108078456442e-05, P60=0.00016614427601809677, P61=0.001669473200855953, P62=0.006369221849433821, P63=0.008215940408059015, P64=0.007988137220872625, P65=0.021293505182785718, P66=0.06547728427647538, P67=0.06362590540583324, P68=0.01532174146679096, P69=0.001860175407539197, P70=0.0010789318640131056, P71=0.001241867627915099, P72=0.002505021282809262, P73=0.003048902518892434, P74=0.006488012953590227, P75=0.008185772519126189, P76=0.004399015594483017, P77=0.005883448282963314, P78=0.0053491299530623515, P79=0.004340573952387193, P80=0.0049552027275027635, P81=0.004782742816086064, P82=0.003770623036165265, P83=0.005697964757017499, P84=0.00993069424258663, P85=0.005705020143957276, P86=0.002994984478676207, P87=0.0028636405738047886, P88=0.003460470855709848, P89=0.003425896117502687, P90=0.0031420522825672476, P91=0.0031528157119362042, P92=0.001886824468148101, P93=0.0010333728310796592, P94=0.0011380324160119397, P95=0.0007548830713069832, P96=0.00033187582492463823, P97=0.0018564168831693006, P98=0.005738852362912867, P99=0.0048205089133234545, P100=0.003182426909160998, P101=0.0012290213511720527, P102=0.0005178369557649381, P103=0.00021913262873993447, P104=0.002117290341176253, P105=0.008406436267191167, P106=0.006926433080124647, P107=0.0026846445749593066, P108=0.0004072315797250948, P109=0.00023947436952785468, P110=0.001540999007236215, P111=0.0021347838075942787, P112=0.0024860673937897515, P113=0.0004713053203846005, P114=0.00023007194605831512, P115=0.00037177856562589537, P116=0.0002219686016598116, P117=0.00021391518173566898, P118=0.00012833343645593564, P119=0.00016504049951025657, P120=0.000251595004235421, P121=0.00021264072771829044, P122=0.00012374487484233815, P123=8.619944728582621e-05, P124=8.633466486054022e-05, P125=6.494387299304089e-05, P126=7.382598792924844e-05, P127=0.00010236356305018457, P128=2.2664109711405696e-05, P129=1.940483310620032e-05, P130=1.685505425570519e-05, P131=1.2100370349054163e-05, P132=1.2282404869605303e-05, P133=2.0397657843833824e-05, P134=1.7007355940454792e-05, P135=3.726988417653934e-05, P136=7.161267763052386e-05, P137=3.4595574809039786e-05, P138=3.5589294985459054e-05, P139=3.5035254297846275e-05, P140=3.829818905508256e-05, P141=3.2110104685461074e-05, P142=4.260328916060857e-05, P143=4.221128324559784e-05, P144=4.471840498987645e-05, P145=3.2189366733911955e-05, P146=3.0602488928914603e-05, P147=4.4547562045655885e-05, P148=7.700961615709694e-05, P149=5.77952857657594e-05, P150=2.126371776779048e-05, P151=1.3024771252059414e-05, P152=9.879797885510867e-06, P153=6.423355197257622e-06, P154=6.427651250733412e-06, P155=1.42937038430955e-05, P156=2.9759640200848125e-05, P157=2.1804788954811614e-05, P158=2.1971439477167243e-05, P159=2.092386660114542e-05, P160=2.1080791452750862e-05, P161=1.9218655344341617e-05, P162=1.940079097411586e-05, P163=1.5741825440160485e-05, P164=2.5022203324346853e-05, P165=2.3964753280352177e-05, P166=4.003424149016511e-05, P167=2.119831055794973e-05, P168=1.8463174011479904e-05, P169=6.577247876025203e-06, P170=5.46081657092717e-06, P171=8.015206915035883e-06, P172=1.4609100236915436e-05, P173=9.584789147427723e-06, P174=1.0537361964315826e-05, P175=1.6854114488034123e-05, P176=1.547247582490166e-05, P177=2.8038707346335074e-05, P178=1.1518572542427052e-05, P179=5.419538516430731e-07),Row( P0=-730.5413752112715, P1=73.93565902480341, P2=8.658458640389112, P3=12.758816669387718, P4=-3.5548351686752793, P5=-0.7160342966008219, P6=-4.638759481316846, P7=-6.551706122990607, P8=2.7386142039662267, P9=-2.6852014294827287, P10=2.6127308399972926, P11=3.5724267249586843, P12=3.4742527089367368, P13=-2.6437285431601523, P14=3.147076636636542, P15=0.28444077709015925, P16=3.769027105624834, P17=6.937220194617042, P18=14.157320634768837, P19=13.483731833768715, P20=10.767136038601432, P21=12.583080612749797, P22=6.747691867054905, P23=2.71627686508729, P24=-2.7441016093439665, P25=0.5407193084743238, P26=-1.958742832073869, P27=0.1959514878419894, P28=-0.7325373577954377, P29=0.429691891950093, P30=-3.905127004470771, P31=0.3627521460497295, P32=-1.0346888257139952, P33=1.1773169531347532, P34=-0.307886477911491, P35=0.20624810577986008, P36=-3.1704770606467694, P37=1.6681465078916868, P38=1.4910093294513858, P39=2.3054895442595766, P40=0.593269515072086, P41=0.5381649676613585, P42=0.5024125049133285, P43=0.5488660344891015, P44=0.5805827883368919, P45=0.5699364272068345, P46=0.6092741392054417, P47=0.6661582310384664, P48=0.7577374734676122, P49=0.7786731227549931, P50=0.7052708874548841, P51=0.6151598221593083, P52=6.836189121631062e-05, P53=2.2620260463725166e-05, P54=2.1253580056762718e-05, P55=3.899736711547264e-06, P56=4.5559736601185506e-06, P57=1.0708998138681724e-05, P58=6.791089204416648e-06, P59=3.041244981494919e-06, P60=8.989792362979698e-06, P61=2.5229189496328604e-05, P62=0.0009072104164431984, P63=0.006650715966018028, P64=0.011858699830800089, P65=0.016209132050553746, P66=0.014520688022798883, P67=0.012867629108916933, P68=0.004357681659232301, P69=0.000433345556224671, P70=0.00013972360822398893, P71=6.289554313314324e-05, P72=8.630692392775715e-05, P73=7.80882535966013e-05, P74=0.00036100163230962573, P75=0.0015550786049692913, P76=0.006043560924567426, P77=0.010708913713213032, P78=0.005940505525968377, P79=0.0041111158999203624, P80=0.0034397858201203986, P81=0.004065276643375339, P82=0.0017653984550139842, P83=0.001176387823767704, P84=0.001294868566899276, P85=0.0003356961099812841, P86=3.9888110117519204e-05, P87=3.568503784574499e-05, P88=0.00013547769319413667, P89=0.0003831013966776143, P90=0.0005141571870898941, P91=0.0005116881478648718, P92=0.00016247582423307457, P93=6.903337602160337e-05, P94=0.00010772095735818598, P95=0.00011883025726382029, P96=5.7702941772444e-05, P97=0.0001582974280389909, P98=0.0006765474738790393, P99=0.0005766024416869995, P100=0.00044560555061738146, P101=0.00023068808161630798, P102=0.00011694002894355337, P103=1.8396127084531617e-05, P104=2.338824126018191e-05, P105=0.0003776167635738337, P106=0.0008682955832771104, P107=0.0002106634824897363, P108=3.0933509648999554e-05, P109=2.1560355307510505e-05, P110=9.530128173902721e-05, P111=0.0002248918915700919, P112=0.00013988251549574144, P113=2.3521434368167995e-05, P114=1.50225948892378e-05, P115=6.227835973628734e-05, P116=7.104970648749289e-05, P117=2.621187262696004e-05, P118=2.38631886296357e-05, P119=9.497544805823987e-05, P120=0.00012038625050000044, P121=5.785498933565415e-05, P122=4.9799643960501956e-05, P123=0.00013198992582124728, P124=4.882895703290961e-05, P125=1.1186331828730715e-05, P126=2.6194145209230518e-05, P127=2.2597531131806427e-05, P128=3.7218352305607707e-06, P129=3.9529144918544725e-06, P130=6.561664929671519e-06, P131=5.982836961436922e-06, P132=9.536288032677273e-06, P133=1.0562016119307347e-05, P134=1.223954447476017e-05, P135=1.6504341777306533e-05, P136=1.5836962144369895e-05, P137=1.4626025488332175e-05, P138=1.4573305754170286e-05, P139=9.69635728993256e-06, P140=1.437786900254561e-05, P141=1.848403556864769e-05, P142=1.7782836540578863e-05, P143=1.2173821370142067e-05, P144=1.2558336225931148e-05, P145=1.0839934129595706e-05, P146=6.93740844996354e-06, P147=8.914446099846284e-06, P148=7.410679011665367e-06, P149=5.7594914687456995e-06, P150=4.852400693292468e-06, P151=5.13177574724137e-06, P152=4.474754314336836e-06, P153=4.399854473123839e-06, P154=3.709754044948481e-06, P155=5.840112504848504e-06, P156=5.113161577648517e-06, P157=7.034646238736351e-06, P158=7.017691063476821e-06, P159=1.016537299871486e-05, P160=1.5447177916274805e-05, P161=1.779879557327959e-05, P162=2.008202646502282e-05, P163=1.0299099588584069e-05, P164=6.215205246554722e-06, P165=6.630801399506931e-06, P166=5.732653869233617e-06, P167=7.011119040425488e-06, P168=5.809533151184099e-06, P169=4.358631688663821e-06, P170=3.3964053570402303e-06, P171=1.018763994774677e-06, P172=1.2605114481077736e-06, P173=5.435888111741557e-06, P174=7.128004187284257e-06, P175=2.8547478749457286e-06, P176=2.3255197078364993e-06, P177=2.4435781569917605e-06, P178=1.8612861201492926e-06, P179=2.3477690948435716e-07),Row( P0=-727.7167721826837, P1=76.41952826671802, P2=8.770170599370969, P3=15.469564992877185, P4=-5.37902148566835, P5=-0.9134024189200055, P6=-2.426241827592006, P7=-4.6310715776354705, P8=4.29083471591567, P9=0.19881256536302191, P10=2.786757474317108, P11=4.533134829722772, P12=5.358784970904226, P13=-1.49381300010079, P14=5.104307042066672, P15=1.0324712822899418, P16=3.613703780070667, P17=6.889027361570234, P18=10.544549725481296, P19=9.346840290751896, P20=7.4836467992842275, P21=10.379162663564665, P22=6.5085082012098745, P23=2.7573628253878124, P24=-1.6757382984112519, P25=-0.5101313314109059, P26=-2.9738588519512366, P27=-0.8816422790661154, P28=-2.6170530575732096, P29=1.2068042685601572, P30=-2.879830177512191, P31=0.6018125932762052, P32=-2.0245094540501243, P33=2.190261499507624, P34=0.39440983159941345, P35=-0.02823661001572982, P36=-4.4003297158581525, P37=-0.2058627537383354, P38=0.8229095019262211, P39=3.38588901480343, P40=0.6564597896028818, P41=0.5739062163461487, P42=0.5701857147126763, P43=0.6094929749892644, P44=0.6070434747826119, P45=0.5868376744292413, P46=0.6620444713789766, P47=0.7394563565298187, P48=0.7918718166938733, P49=0.7349612483180601, P50=0.6686924133911317, P51=0.6466266244713517, P52=0.00010161003126438797, P53=2.9006982896677266e-05, P54=3.332791540603717e-05, P55=1.2926578381219752e-05, P56=6.9768686194985185e-06, P57=7.208286172202752e-05, P58=9.513086633423159e-05, P59=1.890520703001738e-05, P60=8.734297249893724e-06, P61=2.331682129105451e-05, P62=0.0004945319322563743, P63=0.006111929574222766, P64=0.014945094664651435, P65=0.01109787923795694, P66=0.008612412231733584, P67=0.010808182109835647, P68=0.0063042251707497035, P69=0.0014569653345728572, P70=0.0005008897052853118, P71=0.0003893922364949242, P72=0.0002413984718352392, P73=0.0001327973956166949, P74=0.0003863721437055779, P75=0.0011309199702041921, P76=0.005292117064672682, P77=0.011952852434544626, P78=0.007888988070127506, P79=0.002500604948169098, P80=0.0005033171899253743, P81=0.0006783079433382836, P82=0.0011625958249863943, P83=0.0011368667584771906, P84=0.0026841058591176915, P85=0.0021492190080348874, P86=0.00020592063166701772, P87=0.0001606721772814613, P88=0.0005705068252886741, P89=0.0007946503320661184, P90=0.0008570373161652712, P91=0.0003221193638885022, P92=9.12203728346792e-05, P93=4.064420120842555e-05, P94=0.00012903324397714477, P95=0.00019021231062587885, P96=0.0002695574746761156, P97=0.0007864350724985725, P98=0.0004863281483223025, P99=0.00043589412014085623, P100=0.00013676354458044062, P101=0.00010751300334559027, P102=7.131766515746904e-05, P103=9.095055556086195e-05, P104=0.00014005134073265008, P105=0.0007633407796559385, P106=0.0013368533835819733, P107=0.00018279642936689928, P108=6.36415043915767e-05, P109=0.0001365838643458434, P110=9.91188653513077e-05, P111=0.000658269261409298, P112=0.00024343121652025757, P113=4.2700906318871895e-05, P114=3.987701148858817e-05, P115=6.681818914976808e-05, P116=0.00023748629777207102, P117=4.940494308218609e-05, P118=4.1658926659674165e-05, P119=4.396251738312873e-05, P120=3.7395005684173805e-05, P121=9.17315947222076e-05, P122=8.033983256461198e-05, P123=5.026545938824937e-05, P124=4.4810239916023706e-05, P125=4.666722305082585e-05, P126=2.2038225735273892e-05, P127=1.757031485394213e-05, P128=3.348242219915149e-05, P129=5.846468145788429e-06, P130=5.711966136161713e-06, P131=7.107652987489472e-06, P132=6.140875937769711e-06, P133=7.555555466188036e-06, P134=1.3158435545212728e-05, P135=6.159719842761904e-06, P136=8.19510402087515e-06, P137=1.0680182691087625e-05, P138=9.657262793380128e-06, P139=2.3957730437386148e-05, P140=1.921309034717434e-05, P141=2.1898964387157062e-05, P142=2.1587550597144865e-05, P143=1.5043700679475701e-05, P144=1.2235556097261007e-05, P145=1.679890022670537e-05, P146=8.21980047063864e-06, P147=9.49341479292033e-06, P148=8.121968703841616e-06, P149=4.604570656724892e-06, P150=4.7256911307090175e-06, P151=4.808865246406688e-06, P152=3.0857241804183314e-06, P153=2.1548349954328733e-06, P154=2.5114664167841866e-06, P155=2.517719223005467e-06, P156=3.859158200190387e-06, P157=6.5816636468006e-06, P158=8.154895819133283e-06, P159=1.061049761038441e-05, P160=1.2657816761847851e-05, P161=2.1853698573639604e-05, P162=1.52169465972013e-05, P163=6.844457871713428e-06, P164=4.970587295069415e-06, P165=1.0503443526143896e-05, P166=1.3380671970731622e-05, P167=1.0828139190625184e-05, P168=6.358123623610654e-06, P169=4.0110616512314335e-06, P170=2.3027823865529423e-06, P171=2.8558646389818e-06, P172=4.042505425173814e-06, P173=5.4416876305809225e-06, P174=3.584424105901071e-06, P175=1.4455994850994075e-06, P176=4.69219051515666e-06, P177=5.963833629948295e-06, P178=1.4770819547709188e-06, P179=1.3410404560536648e-07),Row( P0=-701.690851805338, P1=78.73100266016418, P2=-1.6012433671664807, P3=5.235869778719951, P4=-8.284598730914324, P5=-3.5360976328002782, P6=-4.351896134855045, P7=0.9981903361512393, P8=4.303391703363491, P9=-1.6777354760241414, P10=0.5601554771397181, P11=5.5636030171185595, P12=1.7288058388351633, P13=1.908451373758303, P14=2.866135798738208, P15=3.3927567125836755, P16=1.8895869799908114, P17=6.95494318500229, P18=8.93702578925689, P19=10.16641254307838, P20=6.253475598057469, P21=7.657986916811531, P22=4.9990054911263115, P23=5.4681322295938966, P24=4.305676996071569, P25=2.8410366041001214, P26=0.17832029111736653, P27=2.2564312753325644, P28=-2.156618112342976, P29=1.283065875536024, P30=-0.1476635862701705, P31=1.8618304852625078, P32=-0.6790912853038498, P33=-0.6124201781337915, P34=-2.537239086930911, P35=0.6598867380427649, P36=-0.8426251300162663, P37=2.0883836922418224, P38=-1.0360034798098479, P39=-0.9908203383690352, P40=0.6854447802141806, P41=0.6561350534446269, P42=0.6075879845293883, P43=0.6099653895886495, P44=0.6713606734278329, P45=0.6971591992515983, P46=0.7635270880630514, P47=0.7420832538301113, P48=0.6971009611024651, P49=0.6947991941073548, P50=0.7140900947171879, P51=0.6853204817282342, P52=7.98939430681748e-05, P53=3.930575682887692e-05, P54=3.412365010997826e-05, P55=3.89313783324555e-06, P56=2.2933457508654143e-06, P57=5.437687008910637e-06, P58=3.1305851182801622e-06, P59=4.721944503711387e-06, P60=9.571025443690669e-05, P61=0.0009392941142041513, P62=0.008055346362176712, P63=0.013245331227666001, P64=0.005329260402996242, P65=0.027392946663583884, P66=0.03998210114701535, P67=0.017389613774356064, P68=0.004402376896947923, P69=0.0012876504251174948, P70=0.0009858794462119674, P71=0.0006165580567773691, P72=0.0009002475062149992, P73=0.0016508604065966128, P74=0.011218983399730898, P75=0.013651700912016033, P76=0.005019021169559055, P77=0.002728400947717638, P78=0.00261542331588422, P79=0.002775472772503086, P80=0.0036582274536755147, P81=0.002735151081024197, P82=0.003554973788337891, P83=0.0037612416778716436, P84=0.0067089680128586135, P85=0.015524825884918507, P86=0.013841324010169185, P87=0.008054502482259059, P88=0.005037244560487922, P89=0.0024121401861953037, P90=0.002171500500650271, P91=0.0023328967337219336, P92=0.0017971302809831407, P93=0.0006941267470548124, P94=0.0009642630552164031, P95=0.0024582621271349698, P96=0.001440133633375005, P97=0.0012074265866794007, P98=0.0008216233167729505, P99=0.0005934135308368243, P100=0.00035710401390602565, P101=0.0026752610836177594, P102=0.0020033768846874707, P103=0.000219942086727274, P104=8.241656338042747e-05, P105=0.0002883913282786648, P106=0.000766400235367714, P107=0.0012115638796293194, P108=0.0010313795904774486, P109=0.0002567216192013621, P110=0.00012210168427774078, P111=0.0002500055714988912, P112=0.0008361878904234998, P113=0.0003473277675139025, P114=6.375521628446693e-05, P115=8.713978166331377e-05, P116=7.905397444246533e-05, P117=7.811113707482691e-05, P118=5.678448476385397e-05, P119=0.00011142347964125469, P120=0.00011725851050341059, P121=0.0001289805240882473, P122=0.00014818021362455908, P123=0.00022808854753184288, P124=0.00016790366291918136, P125=0.0002992503201490855, P126=0.00023116533632926365, P127=0.00010878794708296537, P128=7.27642686835845e-05, P129=4.446484239533733e-05, P130=2.1971293105961435e-05, P131=2.5941328081502437e-05, P132=1.5330807149583225e-05, P133=1.4853995493380868e-05, P134=1.861486485235299e-05, P135=1.4888295574042993e-05, P136=1.7043480421517773e-05, P137=1.8059980352548782e-05, P138=2.496670861414932e-05, P139=1.7225201927013635e-05, P140=1.947772029000921e-05, P141=1.4802858963146782e-05, P142=1.4209917919691165e-05, P143=1.5218225085725548e-05, P144=1.00667711362036e-05, P145=1.915166778587962e-05, P146=2.270740657958118e-05, P147=3.0577500040011204e-05, P148=2.8141896181109933e-05, P149=1.8239628617604977e-05, P150=1.737167665736365e-05, P151=1.3667796855151975e-05, P152=1.12060589245163e-05, P153=9.094555627630126e-06, P154=8.572467757215547e-06, P155=6.270680315779133e-06, P156=1.1825108019908381e-05, P157=1.6307237840666204e-05, P158=1.865851072736385e-05, P159=2.0217058145657254e-05, P160=1.6373872076810367e-05, P161=2.0520502299144915e-05, P162=1.4083403594764483e-05, P163=1.4933222057609455e-05, P164=2.2354356919135485e-05, P165=1.9241502668290443e-05, P166=1.317026012028863e-05, P167=1.7200684307552658e-05, P168=1.5249052281980914e-05, P169=1.870353512511186e-05, P170=1.9018650704313792e-05, P171=4.265583507283097e-06, P172=5.806229642978808e-06, P173=6.709785232561725e-06, P174=1.1316298668700809e-05, P175=1.2629535160072265e-05, P176=1.1811151484069899e-05, P177=5.839683275572801e-06, P178=1.6107170818948964e-06, P179=1.9346975469597106e-07),Row( P0=-568.0153616268814, P1=52.110413059705124, P2=-19.488332548693197, P3=10.324766140623996, P4=-6.673421522711733, P5=-6.6396605747945925, P6=-2.100073414603039, P7=-3.2738215565762583, P8=11.862339664670166, P9=1.8100208837517688, P10=9.623369303684717, P11=16.3595258844436, P12=10.67918525038385, P13=9.929488576528142, P14=13.71536881302128, P15=9.352651034815377, P16=6.409708615339381, P17=9.719532634843468, P18=9.87067571472276, P19=9.434396198999288, P20=0.5266042084101364, P21=3.788289236645808, P22=4.744412011311852, P23=5.748121537804069, P24=-1.0201360148988523, P25=-2.147205244698745, P26=0.49060903844302706, P27=4.769193673384204, P28=3.0575115536516995, P29=5.2655139507585815, P30=1.594981358379286, P31=2.1011908428922497, P32=1.6613693487892582, P33=2.7467011052845316, P34=-1.7572039774482682, P35=-1.9665772894955729, P36=-0.19172115519781593, P37=3.416579712785766, P38=0.23023885416731374, P39=0.4232167640606149, P40=0.5591755105050191, P41=0.6044158273723726, P42=0.6746788256221342, P43=0.5874516633912522, P44=0.5275489740161871, P45=0.5538341470841488, P46=0.6007427687377712, P47=0.64612121784961, P48=0.6843863692795156, P49=0.7264557077213637, P50=0.6543741956954273, P51=0.5681815695171253, P52=0.41485685408360834, P53=0.06170940860919792, P54=0.005444879952029189, P55=0.0003616278941716769, P56=0.00019138484791553605, P57=0.00017977444326591204, P58=6.831397961730768e-05, P59=5.114884979220189e-05, P60=5.529866610557133e-05, P61=7.097500562502562e-05, P62=0.0011277663132028472, P63=0.0049085413254385726, P64=0.008955391377106957, P65=0.02386639421962896, P66=0.03403044725443114, P67=0.04667485021953689, P68=0.20330874183721473, P69=0.6059706965633651, P70=0.5816482031035329, P71=0.14344411016576716, P72=0.03835065749733385, P73=0.051863182919266206, P74=0.16397586588040855, P75=0.25591735888906436, P76=0.1855974831675831, P77=0.08138052285705898, P78=0.014970417284045865, P79=0.007682920262325547, P80=0.008285388453959057, P81=0.008710082988371469, P82=0.012847181987778684, P83=0.018023266037982197, P84=0.03896072596515729, P85=0.048871855222416424, P86=0.04322471269117552, P87=0.05725166805510905, P88=0.036000065631653345, P89=0.01469616232968036, P90=0.00891962471621633, P91=0.010078298550569267, P92=0.006066088842979454, P93=0.0021908308524228987, P94=0.0048563483777008, P95=0.013864042884156968, P96=0.03549956842494174, P97=0.06555826349513491, P98=0.03662387105818809, P99=0.041074795543381865, P100=0.016707179615115583, P101=0.008267873894729686, P102=0.002129063645881503, P103=0.001183607121883891, P104=0.002838063309516696, P105=0.02514902865767576, P106=0.03466467381799215, P107=0.06343175976149994, P108=0.07864684246067592, P109=0.023175318145035425, P110=0.00551247768296318, P111=0.002945873802974389, P112=0.0044914055880648784, P113=0.007585292084618239, P114=0.03277208677869752, P115=0.019015386781824222, P116=0.003354460268870488, P117=0.0020513483319872503, P118=0.009895732522136179, P119=0.005965117064007321, P120=0.006373935843766592, P121=0.001808656975971933, P122=0.002923288097662489, P123=0.008342289320608771, P124=0.002971460423795434, P125=0.003955208845829544, P126=0.0026427792461043123, P127=0.00308271642030925, P128=0.0017355855605908344, P129=0.0032106599720596782, P130=0.0033083779117988, P131=0.0015941036136851534, P132=0.00022780762831163615, P133=0.0003195581987283644, P134=0.0006359789753047254, P135=0.000741450024900368, P136=0.0006309943943491087, P137=0.0005517585727553523, P138=0.0007963735318733618, P139=0.0010947577592483918, P140=0.00045295137981785614, P141=0.0005966308670877484, P142=0.0006651032654733558, P143=0.0007095724545337676, P144=0.000444627546773393, P145=0.0005491983179870944, P146=0.0011394589884156162, P147=0.0003979011919905456, P148=0.00042414514002285546, P149=0.0002031106027833436, P150=0.00023400117874541304, P151=0.0002321548672188415, P152=0.00011591996932563889, P153=7.166890425522959e-05, P154=7.542236045866629e-05, P155=8.430167925832791e-05, P156=7.021615689792538e-05, P157=6.827703779209995e-05, P158=7.69199379597375e-05, P159=5.6436545341543096e-05, P160=6.485018653754296e-05, P161=9.019441556596007e-05, P162=7.851577458164981e-05, P163=5.8435723064519296e-05, P164=4.147869235807259e-05, P165=4.507457581052779e-05, P166=5.3298199925955e-05, P167=5.8080637580790945e-05, P168=5.692173895093757e-05, P169=4.050740333604995e-05, P170=2.990486589132879e-05, P171=3.3415673295084505e-05, P172=4.737091785694807e-05, P173=7.303232806124954e-05, P174=7.069955617329018e-05, P175=9.790573841719907e-05, P176=0.0001228350320177702, P177=9.550488303921938e-05, P178=3.963767008581726e-05, P179=5.185956153566067e-06),Row( P0=-598.1723623972676, P1=41.52944954377707, P2=-15.55846718894544, P3=2.5821072320547405, P4=-7.883772848718407, P5=-12.025046760196267, P6=-4.952252550055855, P7=-8.048913575080064, P8=6.526405183201316, P9=-5.601323809843645, P10=3.4690947381041544, P11=10.942887809391337, P12=6.667253682787871, P13=15.858895747057385, P14=14.3313048750352, P15=13.26638599860424, P16=4.805330198669396, P17=6.12843434589585, P18=4.491153215830431, P19=3.7001229815705874, P20=-1.8071870570590431, P21=2.4769772289220233, P22=2.0426791543542677, P23=1.5673634173178597, P24=-2.4745027701764193, P25=-1.7326780766853405, P26=1.4562911972044745, P27=3.01099249298342, P28=2.602916277186138, P29=2.624862382861365, P30=1.4938393829073506, P31=2.2599248131522875, P32=2.3880945585627664, P33=3.6667063460986427, P34=-0.4178591294651287, P35=1.56032324994585, P36=0.6949581013405893, P37=4.181731521028822, P38=-0.32476288075527215, P39=0.9608160080346112, P40=0.6040231633551365, P41=0.6431704217606075, P42=0.6524436286993148, P43=0.6258858556523117, P44=0.5954453299989091, P45=0.5908248908600753, P46=0.5885861248522197, P47=0.5675800263311556, P48=0.6487693319344874, P49=0.6532597926883764, P50=0.6081564236583002, P51=0.5719258256097789, P52=0.0008268911554763792, P53=0.00021297433606455135, P54=0.00021595206692574765, P55=0.00018374678755477545, P56=0.0001462950229847975, P57=0.0001348087643748711, P58=5.991532939733566e-05, P59=5.649432660829305e-05, P60=9.906923103735389e-05, P61=5.111573405736544e-05, P62=8.637303730834143e-05, P63=0.00021081247161366213, P64=0.0007617922991035867, P65=0.009469448176330298, P66=0.028327603345694397, P67=0.0404047149901733, P68=0.27815983746795064, P69=0.45027273025422865, P70=0.28103768238387833, P71=0.19894225029301554, P72=0.1386601478890424, P73=0.1708947178394291, P74=0.26327602911841286, P75=0.11702259594668607, P76=0.01207950577277118, P77=0.002447631956208214, P78=0.002333221079350964, P79=0.002458540570574116, P80=0.002636248096324815, P81=0.004441870937059056, P82=0.009818737420590197, P83=0.018638269166193096, P84=0.03433767289506426, P85=0.04114622391932336, P86=0.04943906210765343, P87=0.10751737250447164, P88=0.11834222984747851, P89=0.07987064395908994, P90=0.04284974696567014, P91=0.025549044235177166, P92=0.016392572340357535, P93=0.019218779597088703, P94=0.10266565589849577, P95=0.07007458639121704, P96=0.009279475945505015, P97=0.009297235259414387, P98=0.028787340764097592, P99=0.05979480715562177, P100=0.02784221132307816, P101=0.013688874647847333, P102=0.0035014418831513495, P103=0.0031812075595848764, P104=0.009366565526490494, P105=0.022381246593996516, P106=0.028266587848496543, P107=0.05909781841713604, P108=0.019829273567340547, P109=0.03803057392761274, P110=0.08116398188118483, P111=0.035022471313474315, P112=0.0090215206888369, P113=0.003097607790028793, P114=0.013265906331171418, P115=0.013067800227877152, P116=0.012087800369052902, P117=0.008445868031530707, P118=0.005415115106158988, P119=0.006009849645560153, P120=0.005651290768408809, P121=0.00697172302487564, P122=0.004924727285324615, P123=0.006159149137999954, P124=0.0035629492814479454, P125=0.003393664481963717, P126=0.0022526139340868444, P127=0.0016038728173005143, P128=0.0017710624013184352, P129=0.004435986994766842, P130=0.0018325247582756607, P131=0.0012288462963585614, P132=0.0007243169259252683, P133=0.0003588632961605278, P134=0.0003544001252922027, P135=0.0008586727574897127, P136=0.0007408258210642527, P137=0.0006540271888732612, P138=0.0006597228666153363, P139=0.0007229758331027649, P140=0.00048633769180726045, P141=0.0008121705208047508, P142=0.00044437067588602947, P143=0.00025690902636677475, P144=0.00048412621494022266, P145=0.0003844993054870983, P146=0.0004719096173431822, P147=0.0006449585202833524, P148=0.0005824030344821295, P149=0.0004447706959764082, P150=0.00048760730142007806, P151=0.00021415799481516894, P152=0.00022478133672913943, P153=0.0001957003483262122, P154=0.0001351376452579568, P155=7.635539543277658e-05, P156=8.150716890299844e-05, P157=7.311963229218858e-05, P158=0.00016675103937303148, P159=0.00012712209754186927, P160=0.0001391645694810518, P161=0.0002139365763929893, P162=0.00020988939158346736, P163=0.0001419727809079745, P164=0.00014984267327801883, P165=0.00011400256985114283, P166=0.00014441491426424788, P167=0.00023031554741587072, P168=0.00023037807410255623, P169=0.00020935502787417095, P170=0.00018738016053515414, P171=0.00018509047701981173, P172=0.00018127146469346306, P173=0.00020158766778568364, P174=0.00038689348056735644, P175=0.0004295962444329533, P176=0.0003646810080168578, P177=0.0003086398938614973, P178=0.00028387175892971046, P179=2.537788222002531e-05),Row( P0=-730.8789670805545, P1=73.48096609096153, P2=13.487337309840726, P3=12.289935905283835, P4=-2.3486376487094205, P5=-1.2851635033374864, P6=-6.306192331257863, P7=-4.846405993844987, P8=2.571531926208823, P9=-1.6022748973036587, P10=0.9914382677900355, P11=3.7841835415642127, P12=3.5087579500795867, P13=-1.3025955029535559, P14=3.523614998523445, P15=1.954612824530402, P16=4.834117002482796, P17=11.264173303132981, P18=12.997518185569277, P19=11.652473206874044, P20=9.407831694701262, P21=9.196249619264215, P22=4.594823645048155, P23=1.5558709924755805, P24=-2.1976074618385693, P25=-0.1019526325923737, P26=-1.1801912139189827, P27=-1.0013305816124398, P28=-3.8634614916447387, P29=-0.2358720210120646, P30=-4.0292151668560106, P31=-0.4000911540306269, P32=-1.5538387188789742, P33=1.1839228777354374, P34=-0.8487530768619228, P35=1.2496295580500068, P36=-2.252813068036739, P37=2.1139563340104583, P38=0.6291929883716466, P39=1.832687762439001, P40=0.628950105473277, P41=0.557129612231732, P42=0.5778848758024855, P43=0.5887779570460069, P44=0.5936064592957441, P45=0.6099963271694908, P46=0.6523766342068589, P47=0.7012355036852306, P48=0.7870307804520823, P49=0.7769308425326031, P50=0.7081589564991264, P51=0.673316505699547, P52=5.314778824638956e-05, P53=2.382477325722507e-05, P54=2.048431499828815e-05, P55=8.08325090311063e-06, P56=7.5881646767299465e-06, P57=1.0305100832579057e-05, P58=6.529956287576393e-06, P59=9.640544838316872e-06, P60=3.1278120144773715e-05, P61=7.064706252235321e-05, P62=0.0003112816034926037, P63=0.0032069083347349477, P64=0.01393582526395528, P65=0.021679873227770395, P66=0.021971428186002652, P67=0.01707397602762195, P68=0.003797557878666785, P69=0.0007853691963513155, P70=0.0004410787002762027, P71=0.00022012285838096123, P72=0.00016365654960806826, P73=0.00011513358709529124, P74=0.00017515947372199458, P75=0.00030536934656620433, P76=0.0008223998189370332, P77=0.004262949261807185, P78=0.009686175706389369, P79=0.005613002543561572, P80=0.0024582155056190815, P81=0.00178649215128642, P82=0.002026878223179347, P83=0.0009474118343670959, P84=0.00043344589125035896, P85=0.0003212397274894309, P86=7.204564330116069e-05, P87=0.00012047731937513058, P88=0.000563462278005724, P89=0.0003800098572176738, P90=0.0002203028088173617, P91=0.0003446435110269643, P92=0.00016081750885213293, P93=0.00032966160206114283, P94=0.001991155435640028, P95=0.0015901081355968764, P96=0.0002826079247857496, P97=0.00029989167742597734, P98=0.000147595282772184, P99=2.6706275452446396e-05, P100=4.4215366506194164e-05, P101=6.349767200444373e-05, P102=0.0002339049173035707, P103=0.0003447130040464292, P104=0.0001253414857011834, P105=7.835314989363853e-05, P106=3.777536652182397e-05, P107=1.994256558495186e-05, P108=2.9551443376464177e-05, P109=0.0007093695414988811, P110=0.0006439866650888682, P111=0.00013819265066375682, P112=5.747382246124514e-05, P113=2.961035299832778e-05, P114=0.00010816299579801281, P115=0.000623863881151638, P116=0.000267119729119194, P117=4.943706938740333e-05, P118=2.7812980046075467e-05, P119=3.864556452793257e-05, P120=5.249863097837927e-05, P121=5.305893659718858e-05, P122=3.901563732849152e-05, P123=2.3616433316716193e-05, P124=2.106490999672253e-05, P125=2.8062348476016628e-05, P126=9.136293700332687e-06, P127=3.797004270890094e-06, P128=4.360796063713845e-06, P129=2.5049134183320586e-06, P130=2.833824486425558e-06, P131=6.734553812146795e-06, P132=4.075641643129072e-06, P133=2.8511367152916898e-06, P134=2.649240141854337e-06, P135=2.4143311790321343e-06, P136=4.498118023109978e-06, P137=4.525545691941541e-06, P138=1.0254817329801453e-05, P139=8.769799838911695e-06, P140=1.836421243272332e-05, P141=2.3612187015325945e-05, P142=1.3304477727643552e-05, P143=1.2092363490638947e-05, P144=7.795700529987201e-06, P145=1.2955207964905617e-05, P146=1.3439868261269387e-05, P147=1.771911725233222e-05, P148=8.870692776505976e-06, P149=7.59186529897076e-06, P150=6.794636465314917e-06, P151=1.3389613855733926e-05, P152=1.2620707341095265e-05, P153=7.533390308176515e-06, P154=1.2076417318580887e-05, P155=2.67383481499858e-05, P156=2.4658634356133394e-05, P157=3.085238673773638e-05, P158=3.0069421852684064e-05, P159=4.958760902617261e-05, P160=7.658597760506454e-05, P161=6.733791810500134e-05, P162=6.721395946541244e-05, P163=5.618830187668554e-05, P164=2.795942128077313e-05, P165=2.808544660511676e-05, P166=2.871991035667619e-05, P167=4.3961632406123516e-05, P168=4.1282269493388626e-05, P169=4.7095161521081614e-05, P170=3.8550417100390646e-05, P171=1.636442250080063e-05, P172=9.710033450308128e-06, P173=1.947853727217738e-05, P174=3.702468222781254e-05, P175=2.7714905037990354e-05, P176=1.0644608643552628e-05, P177=8.66085492095409e-06, P178=3.973615854554878e-06, P179=4.285573962035675e-07),Row( P0=-484.24278550878057, P1=12.680267382943969, P2=-27.36629713049908, P3=3.636858544376917, P4=-16.37124847838304, P5=-6.67054830708012, P6=0.531874521007504, P7=-2.5286837810919693, P8=15.595571208845818, P9=9.690422523570636, P10=20.873991449387503, P11=23.171556337946935, P12=8.372345537581365, P13=0.1523875574744113, P14=-5.559581300247738, P15=-2.891300001240541, P16=2.1691321512225517, P17=11.112024048108598, P18=10.050365351949239, P19=6.88271526761071, P20=2.3549922191186887, P21=7.356814793947612, P22=6.154343506753284, P23=3.783670926606687, P24=0.14160322786078666, P25=2.4389047344702797, P26=4.11449923004369, P27=6.848057183398412, P28=-0.06656158307896101, P29=3.9829830668575315, P30=4.174929018960493, P31=3.9074622531156398, P32=1.8664017829697936, P33=3.1931411580285016, P34=3.2310341150679838, P35=3.3982379371095273, P36=0.9761888703825355, P37=3.6078859981919176, P38=-0.6603362190992645, P39=2.4031898664252513, P40=0.4591368116325843, P41=0.4429487881657477, P42=0.4389905870337766, P43=0.4488493045334954, P44=0.47121059247274377, P45=0.5177801900398808, P46=0.6162084107628747, P47=0.692849059672454, P48=0.7765638046501645, P49=0.7400647624111598, P50=0.6410898769084825, P51=0.5307471798608941, P52=0.8600112466180004, P53=0.14761949214055228, P54=0.012933350704679622, P55=0.0010630786301373274, P56=0.0007642639858828956, P57=0.00024765469855044783, P58=0.0001650670346524694, P59=0.00013427625926108814, P60=0.00014910124226074858, P61=0.00017566698782699657, P62=0.00021260485299829534, P63=0.00021069284426500476, P64=0.00014546787921948146, P65=0.0001817616104386943, P66=0.00025121855860929185, P67=0.00034481925816014214, P68=0.0006691009561999753, P69=0.0014714703187019623, P70=0.001857036276100822, P71=0.003059761266417543, P72=0.0063358281414506385, P73=0.015201096201348692, P74=0.1345293275243084, P75=0.24891957039577514, P76=0.46817180131173, P77=0.9573821308770812, P78=1.7854871489153978, P79=2.891402943241954, P80=2.720574337712541, P81=1.5718622831121316, P82=1.4729981333370912, P83=0.38746935311105346, P84=0.0434011072817153, P85=0.010998899566126106, P86=0.0022552481345103556, P87=0.0012178619462185705, P88=0.0012464227459550287, P89=0.0007732780778376596, P90=0.0008963524175666065, P91=0.0011255300565221826, P92=0.001042705727569263, P93=0.17174559569449463, P94=4.045625634249906, P95=3.8751864578090256, P96=0.9893343551556294, P97=1.8064000253204766, P98=1.6952865882960855, P99=1.2667961575110924, P100=0.48163189688296565, P101=0.12633552331037948, P102=0.031275927327424895, P103=0.03327783591473508, P104=0.023357371205431184, P105=0.005657078800752415, P106=0.0025410679407194, P107=0.025350841837335797, P108=0.02597581403661555, P109=0.7557474960066214, P110=1.6588880223791829, P111=0.26578081791240754, P112=0.17647211154529532, P113=0.05905329770164508, P114=0.01080118199335389, P115=0.3630908831117633, P116=1.4095098881857624, P117=0.10147423960140185, P118=0.21270494763679534, P119=0.3754893106898376, P120=0.06373027529476795, P121=0.03385840195096798, P122=0.11383251052844354, P123=0.10518107147155722, P124=0.05547736041646672, P125=0.05326635811016998, P126=0.022281075778071365, P127=0.026759781635639787, P128=0.03696189569632802, P129=0.02288658160971447, P130=0.010929884964802513, P131=0.008414455238539172, P132=0.0029925962839375602, P133=0.012171482630700685, P134=0.02067076955310561, P135=0.010003770138164763, P136=0.011110219415412499, P137=0.012059778492038922, P138=0.01710584735285952, P139=0.022465398182334673, P140=0.02161668203471466, P141=0.005675862364282004, P142=0.006234527351481721, P143=0.010548481233917096, P144=0.009960459457238384, P145=0.018281364598046492, P146=0.03793482700898004, P147=0.03936530139044323, P148=0.007844415816972882, P149=0.023338043120596064, P150=0.01614869131180625, P151=0.005849415912044418, P152=0.003303113200986351, P153=0.0010569954654593306, P154=0.001073873954374523, P155=0.0009693999298635567, P156=0.0010506378768523393, P157=0.0013781837036405374, P158=0.0009422474742780152, P159=0.00044279415803997064, P160=0.00039665765430507005, P161=0.00044284640558755426, P162=0.00048174516896255077, P163=0.0004563018278375308, P164=0.0003926651179613556, P165=0.0003013309477100103, P166=0.00022957909470323299, P167=0.00015736809560460349, P168=0.00025473274653840697, P169=0.0003798642094666856, P170=0.0002624231517778098, P171=0.0002664702020091575, P172=0.00020107330871461221, P173=0.00019733222616833342, P174=0.00030393107134266074, P175=0.0005282932934031835, P176=0.000631775113263061, P177=0.0006061631404107282, P178=0.0003580041864926583, P179=4.6337707949839424e-05),Row( P0=-492.98674468358183, P1=15.135712983456692, P2=-26.176398601755473, P3=-6.123936423653556, P4=-14.021795166311497, P5=-8.30366217907571, P6=-3.45896333705987, P7=0.5549143812138025, P8=18.35989596425472, P9=15.016609714140685, P10=16.405452859026838, P11=17.667567284535433, P12=6.151837149415251, P13=0.9987320451759509, P14=-3.106960792851917, P15=-0.05272525602728604, P16=0.32585041729608266, P17=9.493144908584107, P18=8.305174621490279, P19=5.53614794317415, P20=1.857495438801005, P21=8.38924812780688, P22=8.209304993151436, P23=6.400141424038021, P24=-0.5160116126756537, P25=-1.6760280346853278, P26=1.917453377901085, P27=5.8089610862669705, P28=2.1325005097619893, P29=3.6409488651099227, P30=3.7184154203086472, P31=4.377811711298129, P32=2.5098699044758552, P33=2.1447821292807157, P34=1.2520205955594368, P35=1.1789850269403461, P36=0.2538197198470566, P37=3.430621304167854, P38=2.018037431249806, P39=1.945672218368613, P40=0.4983262991554773, P41=0.5181313711503672, P42=0.5162484835670529, P43=0.47782890382789106, P44=0.4687740999622284, P45=0.4910692048956366, P46=0.5360945545611154, P47=0.6265830782560634, P48=0.730281695604533, P49=0.8121201931697692, P50=0.7020709864445253, P51=0.5515881997226728, P52=0.04410737226298067, P53=0.004624636545977003, P54=0.0006131607497547247, P55=0.00015786647312823589, P56=0.00022878962611983897, P57=0.00021292394278875533, P58=0.00010840069549850089, P59=8.787907954083422e-05, P60=9.375882420152133e-05, P61=8.660329796958124e-05, P62=8.637317617210598e-05, P63=9.126889376033075e-05, P64=7.008050806222008e-05, P65=9.30173997611318e-05, P66=0.00013732454388364117, P67=0.00014489255638948964, P68=0.00018778715954893366, P69=0.0002701487729351933, P70=0.0002857134630258278, P71=0.0008425537936680644, P72=0.002182183339493964, P73=0.009582854740797828, P74=0.11441826959833022, P75=0.4823283297838977, P76=1.1309110763909451, P77=1.9877796814094841, P78=2.7097037382089906, P79=3.1544837140720805, P80=1.6738948677166616, P81=0.4635953830218069, P82=0.2913195843113558, P83=1.181866812053716, P84=4.275639298684521, P85=4.625620713453922, P86=2.3941089369527453, P87=0.8878166996719515, P88=0.17634183668713974, P89=0.014738095467043104, P90=0.004887957128402753, P91=0.0038142371351859734, P92=0.01380617247062578, P93=0.3521850527086066, P94=0.8603026674751173, P95=0.4581789168618683, P96=0.6346970460327525, P97=0.9874779950321919, P98=0.26253349265592557, P99=0.07741603059446928, P100=0.08780395136802431, P101=0.04075824094446443, P102=0.008671006903960502, P103=0.003952485533564268, P104=0.010344471572546223, P105=0.11226112112821619, P106=0.46461281610996685, P107=0.8805500230489469, P108=0.5472402289163275, P109=1.1800967046342297, P110=0.9231149843446588, P111=0.34354147622354647, P112=0.20321653484930485, P113=0.07320084672541448, P114=0.062214032138275824, P115=0.39020608961223946, P116=0.15977080292148904, P117=0.08279628121104976, P118=0.03770675714797161, P119=0.018222352722515312, P120=0.03195966900200104, P121=0.03345284172719221, P122=0.02727651980231701, P123=0.010503663943825644, P124=0.00924675515400169, P125=0.007201213708921041, P126=0.005065043650009764, P127=0.004218392414211897, P128=0.0065319615072201975, P129=0.004417483830061815, P130=0.0035033865451646693, P131=0.0027908721282359763, P132=0.011454149957548685, P133=0.025848425756380595, P134=0.003412921830837511, P135=0.0031269105282306906, P136=0.017498090640892434, P137=0.022908727282718255, P138=0.01734843680109453, P139=0.012314158198855993, P140=0.010955132846100692, P141=0.009515236152855412, P142=0.005414874718956984, P143=0.003465899086709335, P144=0.003804098740704787, P145=0.005038057055482025, P146=0.008997076164988975, P147=0.003437084077724596, P148=0.006276270584297738, P149=0.00545158783102239, P150=0.005092534682778802, P151=0.005533585822559761, P152=0.00273807912264461, P153=0.0009496614162611774, P154=0.0006506980232638079, P155=0.0005759044274991281, P156=0.0005950240730420358, P157=0.0006737086288677181, P158=0.0008133605330522535, P159=0.0007309696568704952, P160=0.000591889056612314, P161=0.0006474883983782932, P162=0.0007266737779929547, P163=0.0006059341876719398, P164=0.0005723782904452302, P165=0.0005026432155577726, P166=0.00040858764896269963, P167=0.0002455535624060811, P168=0.00024901328160304787, P169=0.000328201985531645, P170=0.0003310884815564932, P171=0.0005389173716065512, P172=0.0007506159116285616, P173=0.0009046600609517974, P174=0.0005501290070857953, P175=0.0007270596249272622, P176=0.0016128192360592435, P177=0.0016166953241123095, P178=0.0007885001361792785, P179=0.00012042861036348941),Row( P0=-484.3458988426427, P1=14.411687075912416, P2=-24.023503882657188, P3=5.065880150962249, P4=-14.543636973144856, P5=-5.682191988429964, P6=-1.742703427838328, P7=-3.2729990547908696, P8=11.080832595924244, P9=8.556087155964025, P10=17.746922060634883, P11=22.500168938728546, P12=13.926946676710772, P13=5.427185917835544, P14=-3.8417691978900232, P15=-5.092876550150303, P16=-2.391257407968528, P17=8.08564866235339, P18=9.560864727746017, P19=7.616086806105134, P20=2.001023085877131, P21=6.739776257954275, P22=6.019308270267085, P23=6.2282580098187195, P24=-0.537876669579058, P25=-0.25144831860276534, P26=3.403157200870626, P27=6.203261937711504, P28=1.6382752644821261, P29=3.9360255788594487, P30=2.4690010246256904, P31=3.1122665646596395, P32=0.8816247410411551, P33=2.302001418162221, P34=0.8750434441756649, P35=2.0692513019313648, P36=-0.5073278948479975, P37=3.5585969032547786, P38=1.371430406039071, P39=2.0172089373228856, P40=0.48589569518451725, P41=0.4913265686495113, P42=0.4847893722693675, P43=0.5001595732581202, P44=0.541686813193133, P45=0.5825981944393755, P46=0.6425884336459374, P47=0.758406086435715, P48=0.8099775359185485, P49=0.7332110782376382, P50=0.5708596513513499, P51=0.48970219314565455, P52=0.24889572858904724, P53=0.06363264967204514, P54=0.02504341186151959, P55=0.006800260036733267, P56=0.0020146110564066583, P57=0.0005398842449029362, P58=0.00026427820023502513, P59=0.0002039792661205311, P60=0.00041069785566086444, P61=0.00044380214675977196, P62=0.00024215982885507915, P63=0.00020652086443376336, P64=0.00020261421869909559, P65=0.00020486797964845986, P66=0.0002533903997915373, P67=0.0003950842589647239, P68=0.002289691964341363, P69=0.008629101537727535, P70=0.01657309581301614, P71=0.03052468651608349, P72=0.04085420062452913, P73=0.052617949367578555, P74=0.15169299166951572, P75=0.5118076228703129, P76=1.393590293505714, P77=1.4783178417002127, P78=1.6202497201031705, P79=2.267857554672908, P80=1.2332304665123215, P81=0.17154301998018984, P82=0.03459330496967482, P83=0.006027317244345812, P84=0.001981605791119651, P85=0.0011483563716318686, P86=0.0005284114982370199, P87=0.0007838133508991653, P88=0.0011217879687833825, P89=0.0020458688120718886, P90=0.002225821513870699, P91=0.009536740423114401, P92=0.42993377371983277, P93=0.7046513804351573, P94=0.38450412199826633, P95=0.28955963935635964, P96=0.791318491882524, P97=1.3689361245658116, P98=0.7932812366931851, P99=0.49532313255515453, P100=0.05554490121596749, P101=0.028293953133885476, P102=0.0060812160742449465, P103=0.0005409657306253257, P104=0.0012389003340812516, P105=0.009162262481284673, P106=0.006293492890242664, P107=0.025727807811924817, P108=0.16906566351401722, P109=0.5587661873795432, P110=1.43056648007189, P111=4.358290368520914, P112=2.91696060633419, P113=0.08115517612453116, P114=0.3211320988946853, P115=0.06253954149268116, P116=0.00919252106455124, P117=0.05556710186318242, P118=0.13391277706879465, P119=0.066361141538, P120=0.07340196310845341, P121=0.08122232394028059, P122=0.06204756420957984, P123=0.01181853474819805, P124=0.017412229728594096, P125=0.03478828648954162, P126=0.040139402811420234, P127=0.020094587716494642, P128=0.11123967881200865, P129=0.1129659348216134, P130=0.011306644634625251, P131=0.01397223634978982, P132=0.004904525561537935, P133=0.006270937408739795, P134=0.014754574762182981, P135=0.015826870596317724, P136=0.011010088714783066, P137=0.020927633015584347, P138=0.01641065972739755, P139=0.027313798590438892, P140=0.02840088955874108, P141=0.011452271047228246, P142=0.007010746061524123, P143=0.009583784731685986, P144=0.007157640817053737, P145=0.016199025287145518, P146=0.01029381950154808, P147=0.009935778605452198, P148=0.0072391493821523205, P149=0.009840665917777186, P150=0.012501360850503594, P151=0.0034066636538175837, P152=0.0028242670518533017, P153=0.001962701149879678, P154=0.0009712364880400386, P155=0.0008441104936838186, P156=0.0007598924767820871, P157=0.0011421917250359881, P158=0.0010903059798714073, P159=0.0007924061825758398, P160=0.0007084461930444636, P161=0.000609955636623307, P162=0.0004710358454129215, P163=0.00031163138275562254, P164=0.0003020606080350914, P165=0.00024328672339749317, P166=0.00019688797705789138, P167=0.00018943728684893345, P168=0.00025609163690522134, P169=0.0002680810948682709, P170=0.00012199924107853473, P171=0.0001405682117483174, P172=0.0001459952928735463, P173=0.0002648856742548313, P174=0.00027058677667482343, P175=0.00031701528096547205, P176=0.00030836062019331287, P177=0.00033707880999970217, P178=0.00017464142134790216, P179=1.3063305150866337e-05),Row( P0=-645.800138588495, P1=50.66295015031598, P2=1.3777051284930932, P3=-1.519732034636274, P4=-11.337369342107053, P5=-8.114966963065111, P6=0.05623501999327557, P7=0.5465739895616586, P8=0.10470678389816664, P9=-3.7456891289041017, P10=0.0492283499542725, P11=1.3033978170054559, P12=-1.1555192159110141, P13=5.15308540722841, P14=-1.8892425257610381, P15=6.690034799709754, P16=8.339609254366296, P17=12.50843475428312, P18=12.46776323352786, P19=11.15685953791289, P20=8.59570959372393, P21=7.730398030615291, P22=3.17246174412355, P23=2.6246706232017307, P24=-0.8366190627966918, P25=1.1757141232929949, P26=-0.14490334068974825, P27=-1.3629531526050982, P28=-2.5125070674468932, P29=-0.9160688750022201, P30=0.6277267994030699, P31=0.617120464961538, P32=-1.3467893553893613, P33=-0.8594359973239389, P34=-1.7859134435721653, P35=-0.6469619432212759, P36=-0.9765496729151257, P37=0.16009403706644457, P38=-0.9559265163958928, P39=-0.6187975581033404, P40=0.6735029583168569, P41=0.6387672680356177, P42=0.6285289408778733, P43=0.6232571803984711, P44=0.6717979030764066, P45=0.6663457471479128, P46=0.6532325233714437, P47=0.6472915442704275, P48=0.6487913339170174, P49=0.6804187779347656, P50=0.6992150803456892, P51=0.6608700823527801, P52=7.953513346700729e-05, P53=2.3413739532914806e-05, P54=2.443849771511704e-05, P55=9.465975001266782e-06, P56=7.622216813283577e-06, P57=2.5000110832572484e-05, P58=1.1447121081325194e-05, P59=1.054199727860854e-05, P60=0.00010727533383916846, P61=0.0009833748416645024, P62=0.009742690895258643, P63=0.026734231280867347, P64=0.02591679019470918, P65=0.0892244084219817, P66=0.2159641745102213, P67=0.2588658836418465, P68=0.0892477459777193, P69=0.004725146037945647, P70=0.0011213142338746622, P71=0.00037551524634964585, P72=0.0003646533594738144, P73=0.0013858446899022907, P74=0.015555357765374984, P75=0.04574631547270195, P76=0.06283704267519283, P77=0.05142103744834957, P78=0.05079553485705151, P79=0.053438362421039984, P80=0.05584938091769287, P81=0.03231855436637692, P82=0.02196542511778155, P83=0.033929125806593845, P84=0.034671593247991475, P85=0.024743246591542513, P86=0.018247144199061958, P87=0.00848384986111639, P88=0.009307331938113887, P89=0.008945742552011636, P90=0.008164336476844829, P91=0.00945721097029966, P92=0.01828775412569647, P93=0.005183565243719416, P94=0.005794789574295029, P95=0.008757135041839502, P96=0.0037659639898766804, P97=0.0014748823462253985, P98=0.008250054105798947, P99=0.009800973779052544, P100=0.005916277776121582, P101=0.010628595335501657, P102=0.002775353838215158, P103=0.00010214088656364352, P104=0.00017830593726403892, P105=0.0011044018807674328, P106=0.0014470215071567252, P107=0.0015982735067829263, P108=0.002995038566567288, P109=0.0008349810327792887, P110=0.00029232129007667075, P111=0.0005881882128343402, P112=0.0008775208186287658, P113=0.0008574368110246465, P114=0.000458763756995275, P115=0.00018755276915641904, P116=0.0002021639248404849, P117=0.0003347955504113631, P118=0.00033400361999740626, P119=0.00019081051894606236, P120=0.00018308031525019286, P121=0.000710795916521677, P122=0.0020382704750397775, P123=0.0005932761280060139, P124=0.00018754245527573382, P125=0.0003058715342546257, P126=0.0007295324305561974, P127=0.0005845443270280814, P128=0.0002810344379213303, P129=0.0005190859524182385, P130=0.000471209412716948, P131=0.00043122876025289083, P132=0.000442936509613198, P133=0.00013597975605609048, P134=0.00016287369239731115, P135=0.00012312540893684557, P136=0.00020452992299315503, P137=0.00022016044289675826, P138=0.00024838019062644126, P139=9.032867524337695e-05, P140=5.130018486103432e-05, P141=5.841645482804509e-05, P142=4.7381927041357605e-05, P143=4.8990894470698176e-05, P144=4.176549450582886e-05, P145=7.171545372549881e-05, P146=0.0001055362400747025, P147=0.00010288206308331778, P148=0.00014025265721259167, P149=0.00019035955156032695, P150=0.0006012046684255388, P151=0.0004658116101451134, P152=0.00037627627189784515, P153=0.00018324143498331266, P154=0.0001720259189474964, P155=0.00016851730358445174, P156=7.055954400548107e-05, P157=3.99508641898491e-05, P158=3.526854607883331e-05, P159=2.2347506846173248e-05, P160=4.009110359428824e-05, P161=9.956661691102747e-05, P162=0.00011479791082165556, P163=0.0001607272040111852, P164=0.00015409339033195453, P165=0.00017178812733902385, P166=0.0001952417520352459, P167=0.00018506273938430752, P168=0.00018505493988175248, P169=0.00024659840613627744, P170=0.0002636769926082553, P171=0.0005777654422104419, P172=0.0006808902254885536, P173=0.0006287046572606423, P174=0.0007033597822161217, P175=0.0005791032724933914, P176=0.000543770986677184, P177=0.0003119757516173188, P178=0.00012909572297822847, P179=8.623097006281084e-06),Row( P0=-565.3176133605906, P1=40.22712735204255, P2=-28.98961087141453, P3=6.821191612417018, P4=-12.183448542057606, P5=-11.907350446732442, P6=-1.1733369066160049, P7=-6.99079729348465, P8=3.239455816515633, P9=-2.854963500683712, P10=7.106369471149135, P11=9.505026417898584, P12=8.644351653589851, P13=13.370555491167242, P14=13.63451131405666, P15=10.362397612214782, P16=4.284911517382973, P17=7.731090587209972, P18=9.614743878026555, P19=9.591822526026926, P20=7.769978169817818, P21=8.729585406711204, P22=2.944918139569634, P23=4.438632632915192, P24=1.1305359671944102, P25=2.45900659836296, P26=3.178224506102122, P27=0.9486994866343207, P28=-2.9786552128035835, P29=3.116111665156347, P30=0.2913974963384589, P31=0.6416083751375544, P32=-1.1411369873322985, P33=2.1386804126874503, P34=1.5276650724349479, P35=0.3249116727231774, P36=-1.419386585153295, P37=3.388941916601787, P38=3.30163074433342, P39=0.808856901193065, P40=0.5861565156571856, P41=0.5755606328505181, P42=0.5375190837304592, P43=0.5328307387576101, P44=0.5813062577389131, P45=0.6048862060877026, P46=0.580292592147549, P47=0.6034343879199349, P48=0.6231010604891453, P49=0.6525425583343916, P50=0.6439571890727899, P51=0.5889076986114784, P52=0.21345346872062013, P53=0.009073753100099302, P54=0.0001207746667148101, P55=2.2783415279530292e-05, P56=7.219540282570287e-06, P57=4.583643765137057e-06, P58=2.9621659549441382e-06, P59=2.89085526504566e-06, P60=5.3532413051160165e-06, P61=5.848342489783754e-05, P62=0.003169683598066076, P63=0.009586914723925432, P64=0.022585481377532433, P65=0.06453745109508308, P66=0.07997210511304534, P67=0.06662189061483052, P68=0.08620793164635185, P69=0.208524941864868, P70=0.49380330425594926, P71=0.36938687121550606, P72=0.6527313656647621, P73=0.5693888812981692, P74=0.14721232312907248, P75=0.054751802630833735, P76=0.012276619151464642, P77=0.012972415336788478, P78=0.027058367052638633, P79=0.03443957110165792, P80=0.031043518970403618, P81=0.021399521925378924, P82=0.026984851622284017, P83=0.035245814777459875, P84=0.08090865573881241, P85=0.09128601669105255, P86=0.019375414239877653, P87=0.011161579016313022, P88=0.014950434013422912, P89=0.022013770983684396, P90=0.059783031707156764, P91=0.11292801718484684, P92=0.08115646964410225, P93=0.01883674364534853, P94=0.015832389586336117, P95=0.013100974028563547, P96=0.05155879291467346, P97=0.097837736874166, P98=0.05091850671682497, P99=0.03545604809575423, P100=0.04376249712300546, P101=0.02640890477095664, P102=0.025291421387352443, P103=0.022065732722711225, P104=0.05825688206283238, P105=0.04702895529487053, P106=0.06659877086924505, P107=0.03797108383975502, P108=0.015746282128825437, P109=0.011528070588990918, P110=0.12863045371710338, P111=0.1936563079820729, P112=0.1508300339648215, P113=0.05559464382512874, P114=0.00810328896368455, P115=0.0076560031232741676, P116=0.0023635645622983982, P117=0.0078117505276698454, P118=0.01760615988504245, P119=0.03182580177633514, P120=0.01723898657959705, P121=0.010313564160420003, P122=0.0042467006290049115, P123=0.00781340064805679, P124=0.007702843661640207, P125=0.008506525446892038, P126=0.007846790382683669, P127=0.014027639976866752, P128=0.0061615211034065035, P129=0.0019424207367459202, P130=0.011232816561306652, P131=0.007841019531557756, P132=0.003024733865136005, P133=0.0032577176373353818, P134=0.003981740066668077, P135=0.0020226687446485096, P136=0.0017099148237191835, P137=0.005899017918178556, P138=0.010484152265490577, P139=0.005022199694924481, P140=0.0040232101050837, P141=0.004489699883354913, P142=0.0028310805607520734, P143=0.0006643686074914341, P144=0.0014531106654494053, P145=0.0008183019074209174, P146=0.0012169818603274753, P147=0.0012514392302995785, P148=0.0017021772103026566, P149=0.001299183899329141, P150=0.0012261303748972593, P151=0.00041961260443325945, P152=0.00028894348682659846, P153=0.00023645405011803677, P154=0.00014904281895548578, P155=6.974619881734023e-05, P156=7.00592182074381e-05, P157=6.553119613406894e-05, P158=4.339087385906261e-05, P159=4.0995170759132884e-05, P160=3.6057627513274616e-05, P161=2.817362992825115e-05, P162=3.827379350568853e-05, P163=3.4043691840742395e-05, P164=2.630597933085749e-05, P165=2.3303695410121757e-05, P166=2.8745531602064828e-05, P167=3.535291096062488e-05, P168=3.4230472275130207e-05, P169=2.5979225165788063e-05, P170=3.520030994682035e-05, P171=9.484497521953896e-05, P172=0.00021768138697874197, P173=0.0002601475882424374, P174=0.00028621176973727205, P175=0.00037361438315167074, P176=0.0009599141317706218, P177=0.001535230823030271, P178=0.00036422705605142165, P179=1.1604202268778764e-05),Row( P0=-666.3758274810004, P1=60.709463454767565, P2=-11.417288293131882, P3=14.942750475987456, P4=-7.356125869389764, P5=-4.807545902433819, P6=-0.7621612212974631, P7=-1.6393118131148188, P8=2.6380705136728215, P9=-4.566285146174895, P10=3.6140532481862455, P11=2.740759445809277, P12=5.719837253192452, P13=4.318981501404217, P14=7.106790262256191, P15=5.890332118792767, P16=3.980127199502736, P17=7.171685242161848, P18=7.746080460291386, P19=7.886299383876422, P20=5.469416691156973, P21=8.045301086585951, P22=4.289475768725847, P23=4.123918684148243, P24=-0.39995948725402364, P25=1.8183867789865666, P26=0.15849632406363467, P27=1.8141669440086308, P28=-0.8721102438737, P29=1.6719566592615458, P30=0.23772437292862822, P31=1.3688423807077559, P32=-1.341644734658873, P33=-0.19426576341663704, P34=-0.5771401976920177, P35=1.3038926825430628, P36=0.7473615899423832, P37=1.7287272074333577, P38=-0.7658594063028847, P39=-0.05375111787366817, P40=0.6212351336963515, P41=0.6249832464166765, P42=0.6840709559583826, P43=0.6475372994081696, P44=0.6492067310784175, P45=0.6754823820232835, P46=0.7316091315856059, P47=0.7160867692788078, P48=0.6859510349432272, P49=0.6713919703309115, P50=0.6577288086023421, P51=0.6273613890091485, P52=0.002217703374883621, P53=7.307746917777763e-05, P54=2.630677205066096e-05, P55=1.5863739459900955e-05, P56=2.234615739341429e-05, P57=3.529536170306953e-05, P58=1.9920159824035556e-05, P59=1.3077602614527462e-05, P60=4.0604707196220366e-05, P61=0.0001736325986097057, P62=0.0036475510161436313, P63=0.013866228393327415, P64=0.012478067294731636, P65=0.04800357895919296, P66=0.07231596357935192, P67=0.044274709532763086, P68=0.027751418332423044, P69=0.044856902589359475, P70=0.03456831578385186, P71=0.023916033353346414, P72=0.022018928106598057, P73=0.00691270895772404, P74=0.0038421215307963695, P75=0.006148553598199271, P76=0.006366616796074091, P77=0.009157916509058714, P78=0.005444297990086065, P79=0.007278219638782427, P80=0.015374847707629915, P81=0.00713206395702303, P82=0.002692964056518193, P83=0.001831886677634956, P84=0.0018599528106522528, P85=0.0030817449877541137, P86=0.009316321097971304, P87=0.032217381767394554, P88=0.029211638894686344, P89=0.009473707639276255, P90=0.006746369556416357, P91=0.007613531698570641, P92=0.0054106092706016715, P93=0.0035011276493780155, P94=0.004116677313648826, P95=0.004998073741959374, P96=0.005116889372515331, P97=0.009903251453351145, P98=0.010113047898273329, P99=0.00560620575820579, P100=0.0038814099776212055, P101=0.008227661525635295, P102=0.0028096485914245683, P103=0.001132491515646477, P104=0.00520464336699364, P105=0.008886391173570915, P106=0.007214201006553175, P107=0.00639663747125622, P108=0.012328855541438095, P109=0.005927010027947901, P110=0.003507363638877162, P111=0.001921164198015514, P112=0.0013159071672531155, P113=0.0022594959537144644, P114=0.0014424195446423244, P115=0.001660199005919562, P116=0.0013111255352932234, P117=0.0006168166439175987, P118=0.0028918083031746716, P119=0.004035080035010221, P120=0.0022206344265625006, P121=0.0012433184078676925, P122=0.003240352676229573, P123=0.0039337595016433165, P124=0.0011818106138473995, P125=0.001362931258477319, P126=0.0014905400717142123, P127=0.0003169905711678605, P128=0.0002999898481553594, P129=0.0007040723028902816, P130=0.0004547898224865037, P131=8.02589097770424e-05, P132=0.00016931759391948224, P133=0.00026983064049287286, P134=0.00016974911968276705, P135=0.00011207504963016242, P136=0.00016852155426302568, P137=0.00022218669424442142, P138=0.00016999843713985264, P139=0.00019484435184761016, P140=0.0002712683808400212, P141=0.00013988384616841581, P142=0.0001251483599298265, P143=0.00010749513209463548, P144=5.974972698494248e-05, P145=5.25967469712136e-05, P146=5.040694417095913e-05, P147=5.0430349612576394e-05, P148=4.755995309986428e-05, P149=5.5840666772311525e-05, P150=4.570119998788829e-05, P151=4.5744668333574024e-05, P152=2.760090532639906e-05, P153=2.1496249260198276e-05, P154=2.1155548657399515e-05, P155=1.7184731419683734e-05, P156=1.832825558110783e-05, P157=3.0356671232214175e-05, P158=4.269284806097847e-05, P159=3.359074544881027e-05, P160=2.1775454164508712e-05, P161=1.2008524313651214e-05, P162=2.051146339499513e-05, P163=1.0257524358402986e-05, P164=1.1789426235055748e-05, P165=1.3761283685067935e-05, P166=8.796626755505307e-06, P167=6.98139918915243e-06, P168=7.450211970363434e-06, P169=5.450589337493193e-06, P170=2.112594241355009e-05, P171=2.4659132082537323e-05, P172=1.2719279337308196e-05, P173=2.0676103417691586e-05, P174=1.612383678411019e-05, P175=1.4914496831406377e-05, P176=1.800071718047818e-05, P177=1.914057597704807e-05, P178=1.85850517985844e-05, P179=1.4422554519197448e-06),Row( P0=-560.0710251966658, P1=35.535185167377975, P2=-29.57746408268648, P3=3.5335081691202026, P4=-15.320381732452107, P5=-16.10734182842944, P6=-7.078258948672068, P7=-7.085222323288547, P8=6.9587648478391735, P9=-2.226040102897906, P10=10.762096033427081, P11=14.276005056868874, P12=10.766586777899063, P13=11.565066214842021, P14=11.365917435452957, P15=6.5827635539594205, P16=1.8823541702758788, P17=3.960041336162633, P18=7.161875824326375, P19=10.141373839277406, P20=1.8210045426641637, P21=4.729450385516927, P22=4.242290400566531, P23=4.0852875475402755, P24=0.03639793291006861, P25=3.840542868165264, P26=5.030795811707172, P27=5.698994895447377, P28=0.6759564093186545, P29=3.836392793314972, P30=2.4568827193387177, P31=3.1248881034173848, P32=-0.8004095263801424, P33=2.061256653158009, P34=1.9934229919694957, P35=1.3670416967086825, P36=0.13401248969437393, P37=3.457930427395361, P38=3.0371984371093306, P39=3.5175430996772357, P40=0.5261259923395518, P41=0.518002676208329, P42=0.540681616312176, P43=0.566974874545488, P44=0.6158100797541848, P45=0.5806575978021292, P46=0.5555067561541258, P47=0.5619582376892349, P48=0.6517774499971405, P49=0.7069726466862515, P50=0.6394783723350189, P51=0.5557787035694459, P52=0.02943356890622558, P53=0.0017605508041994443, P54=0.00011384329789816604, P55=4.1899449647132746e-05, P56=4.599743356498455e-05, P57=3.280289121409898e-05, P58=1.8841226669905198e-05, P59=1.6020326933058434e-05, P60=2.2513391833349505e-05, P61=2.8439336116966518e-05, P62=9.308225556962577e-05, P63=0.0010972792774049537, P64=0.005084463223928548, P65=0.01913392144000255, P66=0.04068262345364252, P67=0.042709612974171834, P68=0.10714772898680698, P69=0.25464756528511584, P70=0.2066778182027535, P71=0.5904209185987633, P72=0.9462093248379551, P73=0.4663178975686131, P74=0.1547051216010539, P75=0.05133188583643065, P76=0.04206785099697139, P77=0.1303464084381119, P78=0.08630435527325216, P79=0.0288349349057621, P80=0.014690201176641148, P81=0.011531123079210294, P82=0.012243831363072794, P83=0.016569947493384734, P84=0.028640970090169273, P85=0.05121618325572274, P86=0.09795523419476047, P87=0.18043318592815927, P88=0.10798080954656386, P89=0.051065057843742115, P90=0.040261333159250956, P91=0.03769518158146772, P92=0.017393907036687816, P93=0.006014281302936435, P94=0.005835919634244299, P95=0.0025408588924049867, P96=0.0012719593724929798, P97=0.01018799873187783, P98=0.03215910818328944, P99=0.07304324243126536, P100=0.07668750082424854, P101=0.03995708239423328, P102=0.0406606011292412, P103=0.0063379632882753925, P104=0.002647023312938918, P105=0.007291951638712656, P106=0.003183293092857582, P107=0.004376462382737856, P108=0.023557717445886663, P109=0.043841940942759165, P110=0.06431384870647479, P111=0.11598183459518295, P112=0.011386566480682321, P113=0.0007159251891644579, P114=0.001614754610709836, P115=0.005293905183881398, P116=0.005139758970662106, P117=0.008734168742668266, P118=0.009797718706070248, P119=0.004923210101242025, P120=0.00960508123346676, P121=0.013734999938531165, P122=0.007831979500574513, P123=0.004140415455913791, P124=0.00308169647511824, P125=0.0042962694087154765, P126=0.007391181207412588, P127=0.002007091108211395, P128=0.0007770708629411776, P129=0.0015127700099160878, P130=0.0014622555405694598, P131=0.0004236820267755112, P132=0.000599585041632159, P133=0.0007040244685287766, P134=0.0004799108909072665, P135=0.0003794779865385251, P136=0.0010001626275127973, P137=0.0009169912064356836, P138=0.0004824530286145815, P139=0.0005942761560432261, P140=0.0005808479860278749, P141=0.0002978090447406127, P142=0.0003036313954839368, P143=0.0004222830398005458, P144=0.0004992804162679966, P145=0.0003642535687088312, P146=0.00041362060873994155, P147=0.0004887117287714172, P148=0.000326904658976605, P149=0.00026856317144475464, P150=0.00023904760941127957, P151=0.00031321735518816216, P152=0.000129769084185371, P153=0.00017180409382419314, P154=0.0001265465376193877, P155=8.102048528255198e-05, P156=8.462782907990271e-05, P157=8.225164084839651e-05, P158=7.043245884887027e-05, P159=7.057110280209843e-05, P160=6.794063015800487e-05, P161=7.16762280073083e-05, P162=7.182529912682064e-05, P163=8.551529976236807e-05, P164=5.2981037879531915e-05, P165=5.759970714581098e-05, P166=3.390172528898229e-05, P167=2.4112405967931956e-05, P168=2.7591085243578927e-05, P169=3.240166673332196e-05, P170=2.6342975263326043e-05, P171=2.9175417072789294e-05, P172=3.0510590935416964e-05, P173=3.364107389371466e-05, P174=3.971423088233181e-05, P175=4.467637832047124e-05, P176=4.5533349171644275e-05, P177=5.215118766557652e-05, P178=2.6705513025618687e-05, P179=2.743716546405478e-06),Row( P0=-662.4472892767237, P1=54.838656599691944, P2=-10.536768288608677, P3=16.32235277997375, P4=-10.261419286227309, P5=-4.543865808723455, P6=1.5214033610472577, P7=-2.113131415248871, P8=2.521970649770708, P9=-3.233270890768818, P10=2.0843273023124516, P11=2.685210160639183, P12=3.3518794051027005, P13=1.885286440689066, P14=5.028203634060207, P15=3.4421019493725455, P16=2.267794708005168, P17=4.586396444837676, P18=7.589109594042749, P19=6.929146392126449, P20=5.821071481991366, P21=9.324426846676223, P22=5.476401130251671, P23=6.405400169609119, P24=2.41957872082769, P25=1.7801945103905483, P26=-0.7956804666296652, P27=-1.2580336280955091, P28=-4.200292001216199, P29=0.6469467827551603, P30=-0.8061466424056922, P31=0.7754172167310691, P32=-2.1469541378389807, P33=-0.4787785168826995, P34=-2.4012294960000538, P35=-0.23468021273211015, P36=-0.35099939316783335, P37=2.5908464945614162, P38=-0.44246260360408124, P39=-1.1384892314783281, P40=0.6467434960439378, P41=0.6391826605173704, P42=0.5848393447573201, P43=0.5585439273995594, P44=0.6393199792011618, P45=0.7072027107968507, P46=0.7338611618527475, P47=0.713222224740885, P48=0.6852757115940311, P49=0.679726229454207, P50=0.6392213897703101, P51=0.6415509004341072, P52=0.00015592165314196963, P53=3.200938157780741e-05, P54=2.33904990517764e-05, P55=6.949116734595427e-06, P56=1.5581299186616093e-05, P57=5.6528357900905125e-05, P58=2.324034780065557e-05, P59=7.410810728599803e-06, P60=2.1855498439247745e-05, P61=0.0004418324363544802, P62=0.008069639907555722, P63=0.016871354427098854, P64=0.024905251908117015, P65=0.06643668831233396, P66=0.05511670895467516, P67=0.050434617147634704, P68=0.03529302068378311, P69=0.0216299438166873, P70=0.006288862901786907, P71=0.01182071874013713, P72=0.04940961960620319, P73=0.04449446730024117, P74=0.015172082013574926, P75=0.009776517010660605, P76=0.006527578129474721, P77=0.00610677141025551, P78=0.02059852176347541, P79=0.025836406213544067, P80=0.013265053776897965, P81=0.004545501245817144, P82=0.0015040704432589629, P83=0.0018995893810116132, P84=0.005922301340303778, P85=0.012529469422239743, P86=0.012203199597402972, P87=0.006564132520771239, P88=0.0052211285329487765, P89=0.001692312570353316, P90=0.001507741428071524, P91=0.0028599262726846453, P92=0.009218542678071704, P93=0.009029706690680008, P94=0.007375758503230564, P95=0.006389960022730511, P96=0.004499028410052121, P97=0.0063799400162159285, P98=0.0034449735482233197, P99=0.001237340904381032, P100=0.002320639126444662, P101=0.01663376850721027, P102=0.007536480535727641, P103=0.002514643446720868, P104=0.0027055029053604965, P105=0.001757558752636387, P106=0.0011637117315242472, P107=0.003868467356113912, P108=0.02545834362872271, P109=0.007048685088253411, P110=0.0012940357238483185, P111=0.000551484907850778, P112=0.0004101846876820746, P113=0.002820542700697143, P114=0.004109710944772626, P115=0.0013475341679477346, P116=0.0003899354468242849, P117=0.0007163142201008994, P118=0.00881015490182462, P119=0.009801881296037636, P120=0.001819565045061039, P121=0.0014015739669508316, P122=0.003267269325238492, P123=0.0028269001306184777, P124=0.001371536866723273, P125=0.001306328982495651, P126=0.0007579109123453633, P127=0.0005710916807641257, P128=0.0006623060191765924, P129=0.00039905239161400517, P130=0.00040398261556089696, P131=0.00041506540964013583, P132=0.0001580112578491901, P133=8.854660681290329e-05, P134=0.00012515465657464192, P135=0.00031456607789415804, P136=9.010804404710701e-05, P137=0.00030711636792598966, P138=0.00021551777460761633, P139=0.000258478693323635, P140=0.0002786401620612744, P141=0.00022997302799791693, P142=0.00022083062477517423, P143=7.147628004576065e-05, P144=6.242949605435038e-05, P145=4.600282466625385e-05, P146=4.2498199464641966e-05, P147=4.70727650995055e-05, P148=4.4252619946622184e-05, P149=4.155106509811955e-05, P150=4.090914990561723e-05, P151=3.7520725693636985e-05, P152=3.2739646650264314e-05, P153=1.646765334981794e-05, P154=1.6075391651482595e-05, P155=1.804668901401222e-05, P156=1.691607557507035e-05, P157=3.1011125326430086e-05, P158=1.840549483554547e-05, P159=2.1635666934312866e-05, P160=4.1022897294556434e-05, P161=2.6634408156349582e-05, P162=2.4254157210160975e-05, P163=1.5120900364046463e-05, P164=1.0118689166786003e-05, P165=1.0806306641297702e-05, P166=1.015912504289263e-05, P167=6.968177448128013e-06, P168=6.8944040448439864e-06, P169=8.110109244771173e-06, P170=6.575236402212155e-06, P171=5.7138989434105295e-06, P172=4.746186492254593e-06, P173=5.159596595211006e-06, P174=6.759971358066573e-06, P175=7.158979846601793e-06, P176=6.927952246259469e-06, P177=1.3404328048906029e-05, P178=1.0547737203753577e-05, P179=1.2180515871687885e-06),Row( P0=-482.3245066507177, P1=13.467038087994462, P2=-22.57886962192694, P3=0.14357300723534197, P4=-12.42299155630396, P5=-6.029165020640856, P6=-1.0683782611296924, P7=-3.3794048964548478, P8=17.795102489079785, P9=10.901889797182665, P10=21.272896370631262, P11=27.808137337466313, P12=16.67442838096711, P13=9.847226977219824, P14=-1.4089478243782303, P15=-2.394541710089742, P16=-3.788315017517087, P17=8.137681740451638, P18=10.325911673406976, P19=7.730282807950013, P20=1.2784291515135746, P21=5.521241866644255, P22=8.51044019848469, P23=10.31962253188509, P24=0.9245250759070002, P25=-1.1352329868734394, P26=2.3086184412923254, P27=6.751276300068774, P28=3.374788660360459, P29=3.5123018020942407, P30=2.4806143537590173, P31=4.257251697253999, P32=0.9769121656245624, P33=2.844504806910708, P34=1.9677758314322937, P35=2.9718802031626326, P36=-1.7289318981749389, P37=2.860292555252407, P38=2.699384377124196, P39=2.437646632446521, P40=0.4230317793819202, P41=0.4328912828086926, P42=0.42863996129175425, P43=0.4825817751406137, P44=0.5766308984868312, P45=0.6032200967182815, P46=0.6565019486903728, P47=0.7347927080849773, P48=0.783304945157276, P49=0.599959419009842, P50=0.4730651042945654, P51=0.4470987377838585, P52=4.83010221579658, P53=1.7248139488828167, P54=0.15748337442152316, P55=0.016332351066409408, P56=0.0025714146834376916, P57=0.0007372878221783444, P58=0.0003885363017800272, P59=0.0002992133252766072, P60=0.00023827442174808168, P61=0.00013668160759704962, P62=9.703861664871553e-05, P63=5.689727587000947e-05, P64=3.1879638370558276e-05, P65=5.271771161943058e-05, P66=7.967670961399293e-05, P67=0.00014024581649032825, P68=0.0008831695554006067, P69=0.002741630406335259, P70=0.004818072418827771, P71=0.02450372536125106, P72=0.11644548808679439, P73=0.18998949894292574, P74=0.6305272933391888, P75=1.1165827865834093, P76=2.4352856719007865, P77=7.2802254038216825, P78=5.88339946534141, P79=2.3636209955844336, P80=0.6657121852157686, P81=0.19830904289712606, P82=0.1513069214147313, P83=0.07690382179613209, P84=0.033709389695174855, P85=0.018625373259013922, P86=0.004168860189111478, P87=0.0028031947987906163, P88=0.0016014557853645114, P89=0.0011461903896356027, P90=0.007818464228648405, P91=0.31697739533397173, P92=1.3939992591422699, P93=0.6995283179704969, P94=0.820762259495796, P95=2.1243722563393255, P96=1.5291613810665032, P97=0.7217421762362589, P98=1.6083857161230422, P99=2.7843995915818485, P100=0.1448732719828021, P101=0.00476908416294006, P102=0.0011255015874490322, P103=0.004542511898279224, P104=0.019124276677113118, P105=0.03291087890227923, P106=0.019195473595093358, P107=0.6688530834983358, P108=1.6912217721821623, P109=0.2622015492323556, P110=0.1829030797616107, P111=0.07226362959158218, P112=0.01937954939404513, P113=0.44794013187016524, P114=0.471291660493726, P115=0.04607629441691302, P116=0.03747676848342638, P117=0.11870202247643032, P118=0.0738384641584205, P119=0.0734096653723802, P120=0.19205275574468117, P121=0.04985737253202907, P122=0.013970676135444187, P123=0.014783264790794727, P124=0.03788788149490567, P125=0.04221915600805793, P126=0.015149398049132169, P127=0.0114649985985008, P128=0.011172315249180938, P129=0.006684444802190483, P130=0.011172113230388313, P131=0.03218355963944282, P132=0.09395339388381527, P133=0.06733357913147064, P134=0.004330910261907397, P135=0.005478289471516966, P136=0.011319739948433617, P137=0.03140759508629824, P138=0.019365795261686605, P139=0.009270733143750494, P140=0.02026247771254233, P141=0.020539450661881616, P142=0.018376624727125732, P143=0.002917857531015302, P144=0.010213238862362849, P145=0.028663675047929448, P146=0.008021276694965163, P147=0.009130580416892322, P148=0.029905923577335923, P149=0.013881449815605308, P150=0.008416762276973768, P151=0.015930512639061156, P152=0.005017841730546805, P153=0.0024337940595623624, P154=0.0016318339085216375, P155=0.001340808906278669, P156=0.0009832517178239776, P157=0.0009567360499653291, P158=0.0011339326185513174, P159=0.000816835263520629, P160=0.001185953051122304, P161=0.001055091758725896, P162=0.0008588907733065806, P163=0.0013125732232594002, P164=0.0009653223218252422, P165=0.0006499592628265741, P166=0.0005717020410150522, P167=0.0005199148672032515, P168=0.0007903025102051377, P169=0.0007409780634424813, P170=0.0006379867667970276, P171=0.0008733855051485459, P172=0.0012705077465755923, P173=0.0014344681681907623, P174=0.0011988506183604095, P175=0.002268386738984544, P176=0.005086731410033726, P177=0.004919576516942566, P178=0.002631509205398599, P179=0.0002475557275430338),Row( P0=-661.2293403898125, P1=75.0361822633786, P2=-7.4743936497365, P3=1.5447491415282992, P4=-13.929053623352884, P5=-8.750948759182192, P6=0.07331998282993663, P7=0.1515316313703699, P8=-2.42019155384097, P9=-4.951225162418109, P10=-0.8877508817149546, P11=0.01737235039803461, P12=1.190969196010864, P13=4.8011670009113185, P14=2.5905149869416895, P15=7.095827266460989, P16=6.566662009808708, P17=9.285417768381839, P18=11.563806147886881, P19=8.11921424512127, P20=8.226651512473047, P21=8.735212309747935, P22=6.79504003525351, P23=6.3644426593312495, P24=2.219566878140392, P25=4.293583734433908, P26=-2.0048897804805255, P27=0.3785387805219852, P28=-0.9415067659878931, P29=1.3659350022561207, P30=-0.9588139140469808, P31=0.4729627824519983, P32=0.10953894219730287, P33=-0.22142336633355367, P34=-1.4347358040030398, P35=0.29042471722347707, P36=-2.127331899825094, P37=0.21978234427213747, P38=-1.6804284766830913, P39=0.9642552491133092, P40=0.7011021187109601, P41=0.6499651645814842, P42=0.6165521865790752, P43=0.617409446864936, P44=0.6379241997091115, P45=0.6769431708374155, P46=0.6754376579525898, P47=0.6883066995274072, P48=0.6512571290010923, P49=0.6413970117886786, P50=0.6236575894561592, P51=0.645439799989911, P52=0.00010645125804130507, P53=2.756266079452038e-05, P54=2.7136412921240666e-05, P55=5.278989752846093e-06, P56=2.1174538754230596e-06, P57=3.0430829393953655e-06, P58=4.196924545866292e-06, P59=6.692024065638291e-06, P60=3.431423916581561e-05, P61=0.00037343068582549487, P62=0.007260051570291178, P63=0.010964305262250335, P64=0.007351788306957985, P65=0.017741705071763286, P66=0.05571984543647066, P67=0.0905240986274254, P68=0.032858233955607954, P69=0.005012197092457996, P70=0.0024963367710759696, P71=0.0009274465223264249, P72=0.0009450878431553873, P73=0.0016660550217065429, P74=0.004645397853640471, P75=0.007712386833889027, P76=0.009135433451783378, P77=0.019987031609086214, P78=0.015752944104690345, P79=0.00966810671506049, P80=0.0074600580765527315, P81=0.0067107691725410154, P82=0.011484971087180794, P83=0.017367361522810033, P84=0.01978042600306854, P85=0.015171255001272697, P86=0.008033870548842942, P87=0.008380589124893834, P88=0.014066364385904973, P89=0.02668517816642965, P90=0.03015748871946161, P91=0.014973619705344563, P92=0.016507548148782438, P93=0.00944911012583165, P94=0.00461366340061943, P95=0.005039497789033701, P96=0.0037581656151643504, P97=0.0016320768420112854, P98=0.0021121239709885214, P99=0.002445239391572134, P100=0.001303833235217156, P101=0.0023996577886774397, P102=0.0012332134362037694, P103=0.0003315993655052585, P104=0.0008956580611307768, P105=0.0009029717657171994, P106=0.0006848082718314071, P107=0.00043932597987741376, P108=0.0009080526770676787, P109=0.0009026591207151188, P110=0.0006607070819119696, P111=0.0001492875652960306, P112=0.00010902608349437599, P113=0.0002584304100608788, P114=0.00013875283251368883, P115=0.0001335057363776581, P116=0.0001829359047561546, P117=9.141357626249227e-05, P118=0.0001029426263714871, P119=0.0002060257363640404, P120=0.00017378608726957405, P121=0.00027487340365787695, P122=0.0002800933688889953, P123=0.00018499625257499, P124=0.0004045333672916507, P125=0.0005079647043202402, P126=0.0003820038849867697, P127=0.00013159128579018055, P128=0.00014822873071024492, P129=0.00012615042223018444, P130=0.00019921656302889304, P131=0.00015500609710639325, P132=8.82333437647882e-05, P133=9.815479874552955e-05, P134=0.00010495222424220486, P135=0.00012100274528928885, P136=9.221984737625042e-05, P137=9.551785436918022e-05, P138=7.904346399965605e-05, P139=5.917542602124701e-05, P140=3.9835942885604026e-05, P141=2.9571901291521293e-05, P142=2.91847828600973e-05, P143=2.5268174148119776e-05, P144=1.6636362116871097e-05, P145=1.4119726607797262e-05, P146=1.9734783319772566e-05, P147=3.2016094966664144e-05, P148=5.3742870629547265e-05, P149=9.099224548893685e-05, P150=5.346968545586362e-05, P151=2.079826240821334e-05, P152=2.6823616813236108e-05, P153=5.1039298643638584e-05, P154=4.371863273298867e-05, P155=3.5917158384335387e-05, P156=2.7721897623997227e-05, P157=1.2644048760427363e-05, P158=8.494792123404632e-06, P159=8.237517205190523e-06, P160=5.081018919934419e-06, P161=1.572637908806686e-05, P162=1.8354143958339516e-05, P163=2.8187393375373325e-05, P164=2.896571238450169e-05, P165=3.105613227765091e-05, P166=2.4423168477270935e-05, P167=1.553818596253908e-05, P168=1.663685274193511e-05, P169=1.925424731283721e-05, P170=1.196089432048972e-05, P171=9.309088371637531e-06, P172=1.1400858983535832e-05, P173=6.165977278504246e-06, P174=8.096153526395603e-06, P175=1.2696011250610464e-05, P176=7.857873299919456e-06, P177=2.9323462816601223e-06, P178=1.5461206510335293e-06, P179=2.249002095009888e-07),Row( P0=-568.9737691497037, P1=38.69021365165977, P2=-31.724772342430466, P3=4.000852238042293, P4=-17.512185798284865, P5=-16.523422010303246, P6=-4.353485424511485, P7=-8.735911527831277, P8=0.16099241950381427, P9=-4.6511527511028445, P10=4.264953035973778, P11=9.079496793587907, P12=5.658639147388073, P13=12.9065030869, P14=13.406688952611201, P15=14.214405697062269, P16=6.608535139810828, P17=6.9227924910968675, P18=6.660442567501321, P19=5.056433721850128, P20=5.044652573601815, P21=5.708058440112648, P22=1.208267690292372, P23=2.9693284674963554, P24=-0.9081629213678565, P25=0.4411228799682003, P26=4.223834796660988, P27=3.5407427690817928, P28=-1.3364075671090634, P29=1.1587651799507552, P30=0.7175559274640083, P31=2.5380410347866746, P32=-0.03511000302556658, P33=-0.08349270279033935, P34=-1.3142870687525186, P35=1.6729107578763818, P36=0.48691423443496146, P37=3.60051287521391, P38=-0.3871052738448033, P39=0.09564492181106313, P40=0.5897875356109153, P41=0.602666717158399, P42=0.6118037256433679, P43=0.5552514442530513, P44=0.5650858260243273, P45=0.5751254783386041, P46=0.6019976920141403, P47=0.5915572640345961, P48=0.6371354964639742, P49=0.646335538453038, P50=0.6288251207275457, P51=0.5779430044839141, P52=0.0005688281054177839, P53=5.184976097331217e-05, P54=4.1474901521512054e-05, P55=5.398741373537896e-06, P56=3.913679100305254e-06, P57=5.3381213006414356e-06, P58=3.424114812713803e-06, P59=1.947282578180132e-06, P60=7.1355179993323904e-06, P61=0.00014869437023951008, P62=0.0030011397018516444, P63=0.005054897504580014, P64=0.006552780703897597, P65=0.029332422237763, P66=0.06960161490533594, P67=0.06762006946896859, P68=0.12023085715729977, P69=0.5771934056249407, P70=0.9378513352102935, P71=0.3452032391834528, P72=0.23144922264666226, P73=0.1348192262593009, P74=0.043622272442771996, P75=0.016838043712242214, P76=0.005846199150714872, P77=0.005063566099841046, P78=0.004935465369239017, P79=0.0070798362301292845, P80=0.026345243083734003, P81=0.04804664380822148, P82=0.04085384551257654, P83=0.025612401923866122, P84=0.03884236803077196, P85=0.05411404284339805, P86=0.02967846558043523, P87=0.04728865318262119, P88=0.07827470751644719, P89=0.030588785357335863, P90=0.02956852104168459, P91=0.03921456647675174, P92=0.034775146755459806, P93=0.0248982784256092, P94=0.024361031801169934, P95=0.014431039833306358, P96=0.004633133303126284, P97=0.05555050146661461, P98=0.08758527371432358, P99=0.03290449758439212, P100=0.27708532002488634, P101=0.16969005075818688, P102=0.03318756793156583, P103=0.01291311738036487, P104=0.0022866961071427478, P105=0.0018879687516512058, P106=0.001982360963641507, P107=0.00289282868441992, P108=0.00315301741763339, P109=0.024977184530226872, P110=0.02986804830481154, P111=0.004955662202498442, P112=0.0027843394340260545, P113=0.003542224109966983, P114=0.005287674257466583, P115=0.006801610598341685, P116=0.039785969059773184, P117=0.00868181014900587, P118=0.004621532566333398, P119=0.007681528013782456, P120=0.004976054300676824, P121=0.005136835555209279, P122=0.0033449242797796257, P123=0.0033653023886865557, P124=0.005619307252702148, P125=0.0051369660762355315, P126=0.0034592555698199348, P127=0.0023538754614428775, P128=0.006980009221469765, P129=0.015158743821760396, P130=0.007642868386916214, P131=0.0031783629405674864, P132=0.003452268827600364, P133=0.00234918965983065, P134=0.005019952411623926, P135=0.0025729805654373652, P136=0.001226954126635508, P137=0.002737390598097884, P138=0.0026307693153229503, P139=0.001462165335886771, P140=0.0031823584204972327, P141=0.001158132800497686, P142=0.0004028278534065563, P143=0.0007888251165155106, P144=0.00035745161433484336, P145=0.000845761445458094, P146=0.0006798297691711766, P147=0.00045334743952996163, P148=0.00038285672677299893, P149=0.0003335750760673497, P150=0.0002551027931953408, P151=0.00022422475666233026, P152=0.00014613584941592349, P153=0.00013168871139014582, P154=6.285385088975823e-05, P155=4.390103974944086e-05, P156=3.719056629333412e-05, P157=4.7711512302019225e-05, P158=7.090739342637042e-05, P159=7.212456909487615e-05, P160=0.00011063523412129413, P161=8.668850661541675e-05, P162=0.00012118676840885418, P163=0.00010342068183077693, P164=7.89954716173644e-05, P165=9.596285573774712e-05, P166=0.00013495228698079197, P167=0.00019368060331471015, P168=0.00017352093284239413, P169=0.00017266077251620242, P170=0.00014648536121988855, P171=0.00018654435706921122, P172=0.000137492302074753, P173=9.505457409433915e-05, P174=0.00011399206629342724, P175=0.00016868789284621923, P176=0.00046680175339883337, P177=0.0003046443251410834, P178=0.00016635030625096025, P179=1.813648176088321e-05),Row( P0=-561.3205533357506, P1=36.55444027358618, P2=-24.181813494646626, P3=12.486649596703135, P4=-11.271593706582431, P5=-6.894786348110513, P6=0.009583834531136202, P7=-6.952123527334215, P8=1.0923544854267055, P9=-1.591077661925667, P10=3.9588774132294855, P11=4.914081018040226, P12=5.54730253954365, P13=10.52686495585243, P14=11.267042603088498, P15=11.161317616263789, P16=5.869429174776787, P17=8.916257909298164, P18=10.050078520111628, P19=8.719792499213524, P20=9.046646435170182, P21=9.315087366461226, P22=6.045102977400535, P23=6.6798638310302625, P24=2.800144147254285, P25=2.8820983912906892, P26=3.275766998122375, P27=3.1402402104896785, P28=-0.502823179034499, P29=3.8048023240833486, P30=2.540012598023786, P31=2.763102954163896, P32=0.6603615852030712, P33=2.46959325325221, P34=-0.16168410522236032, P35=3.209310915045886, P36=1.7186230036148447, P37=3.058042678150777, P38=-0.29242332669526866, P39=-0.09122809079855407, P40=0.6202710815030841, P41=0.6405828224160514, P42=0.64900154875183, P43=0.6206370200351814, P44=0.6264212578008402, P45=0.6459529053437966, P46=0.7018886240982378, P47=0.6594411549022313, P48=0.6476585556734261, P49=0.6698370144148089, P50=0.6681768934542758, P51=0.6127335877147204, P52=1.095066883950374, P53=0.10618351437668408, P54=0.002307278428814288, P55=0.00011350790258750976, P56=4.88598696305886e-05, P57=1.2045650908026886e-05, P58=5.988982039895867e-06, P59=5.898631009829409e-06, P60=1.408900907683794e-05, P61=0.0002208598523081246, P62=0.011907425312836313, P63=0.028454471640346323, P64=0.03171079585904617, P65=0.0678632089751801, P66=0.09214748028148875, P67=0.08137163319459338, P68=0.1986754525711127, P69=0.4556049661537655, P70=0.2523881921357115, P71=0.21529918383190494, P72=0.25382775499460014, P73=0.14611433541634858, P74=0.04517194330887136, P75=0.018499048558330195, P76=0.00859925796044351, P77=0.01781878977795361, P78=0.03790586184470077, P79=0.043781266833669855, P80=0.058388073987136274, P81=0.04722795196314264, P82=0.02229536242594381, P83=0.006203235806159637, P84=0.003532672871338096, P85=0.004015764295243349, P86=0.004411534093144322, P87=0.010558138140458022, P88=0.03239841259203853, P89=0.05514401501553301, P90=0.05701051762562241, P91=0.05618621607980974, P92=0.02064951096525316, P93=0.024397332213481508, P94=0.06522925267446161, P95=0.037135201019641344, P96=0.01569790386431752, P97=0.03646295872941, P98=0.0201370127629159, P99=0.013398326588460602, P100=0.017758884186176233, P101=0.01793696098011213, P102=0.04319328113084651, P103=0.034062215689113606, P104=0.005127574345999966, P105=0.03987040619904096, P106=0.06131389881484346, P107=0.03288557206925608, P108=0.03273420613568004, P109=0.027010800907594928, P110=0.024469942471249066, P111=0.02041418057199185, P112=0.028983992555450935, P113=0.02805339169253073, P114=0.008147446326796036, P115=0.010041038181652725, P116=0.005557659229610381, P117=0.009677657502331604, P118=0.018942391462745952, P119=0.01224286858374512, P120=0.008518883446134108, P121=0.0044087051934357125, P122=0.003872566433809912, P123=0.007470311501774417, P124=0.010128302293559549, P125=0.002357840535059584, P126=0.0011148067516097925, P127=0.01288902268271153, P128=0.014421897319115065, P129=0.007246305415707046, P130=0.006931582039982234, P131=0.006081418314740169, P132=0.003285813762981671, P133=0.0017293050243527629, P134=0.0021064545652406358, P135=0.0015500619996095697, P136=0.0055886435924144716, P137=0.004515768066818712, P138=0.004813252749618192, P139=0.00735527986432051, P140=0.004008415065393592, P141=0.00268540383603281, P142=0.0009829819385274714, P143=0.0009314912083018647, P144=0.0006462562506210096, P145=0.000643126876658365, P146=0.0008172909488282248, P147=0.0010958298348643705, P148=0.0008847815646831635, P149=0.0007247973807845296, P150=0.0004960857015985713, P151=0.00038134714853809863, P152=0.00025338393751136453, P153=0.00028003561425851253, P154=0.00015479262437780917, P155=6.677954891538728e-05, P156=6.567544946254376e-05, P157=4.970787435622391e-05, P158=5.405343349517218e-05, P159=4.8646622181505916e-05, P160=6.823409976159024e-05, P161=7.02042253882464e-05, P162=5.0924996784681836e-05, P163=2.9494148097853943e-05, P164=1.469376522314049e-05, P165=2.6768109138806986e-05, P166=3.883322341976568e-05, P167=6.311417932628277e-05, P168=7.242787689378025e-05, P169=5.539107875176248e-05, P170=3.359326697356451e-05, P171=1.922553229164559e-05, P172=1.6626576393625306e-05, P173=2.5064158513100536e-05, P174=4.8142572203715787e-05, P175=0.00010638467237520592, P176=0.00011943876362896576, P177=0.00010396945968181242, P178=4.7079110151496466e-05, P179=4.3060720768449654e-06),Row( P0=-682.96145108948, P1=64.66049782139889, P2=6.127560902026649, P3=0.46564205019208205, P4=-6.742156511240024, P5=-6.879398085494806, P6=-3.6242776538037313, P7=0.8608900019615624, P8=3.5369214755092586, P9=-4.293896649620155, P10=-1.0804652555889125, P11=5.4457958402446085, P12=-1.2296250597550717, P13=2.5111522155421784, P14=0.8650237788980683, P15=3.7546910260511175, P16=2.9954134890166304, P17=9.313333705637872, P18=11.144435550424518, P19=13.982397802201927, P20=10.619878732558979, P21=9.378755506346213, P22=2.7764482883046204, P23=4.161903036147777, P24=0.06057525215797605, P25=-0.4172716168290139, P26=-0.29287340968338277, P27=0.9878600804437627, P28=-1.3048209511894857, P29=-0.7079582509139173, P30=-2.0637624587911225, P31=0.7394368491219372, P32=-1.0653461954953882, P33=0.5530529709785686, P34=-1.5438551613452332, P35=-0.2773820117842144, P36=-1.5377847545190748, P37=1.9713127788770808, P38=0.2307006292536232, P39=0.14813604520346568, P40=0.5658601038668768, P41=0.5285543496747942, P42=0.5473162163890369, P43=0.5944608546347642, P44=0.6259309344138246, P45=0.613944331483108, P46=0.6714147863997615, P47=0.6879228910162752, P48=0.6788896871718334, P49=0.649892114597918, P50=0.6324319744841446, P51=0.576116525591776, P52=4.007496591420984e-05, P53=3.41818025199815e-05, P54=3.742456536458709e-05, P55=4.611916896503587e-06, P56=5.353112437124102e-06, P57=2.133927559591622e-05, P58=1.3867329875552109e-05, P59=2.1704759941422044e-05, P60=0.0003384392040516143, P61=0.002000147333364141, P62=0.006791078025470985, P63=0.017255085517548487, P64=0.01985133221894891, P65=0.06349306019237264, P66=0.065803358560767, P67=0.025776334603123414, P68=0.0064947840746153425, P69=0.001333006287511241, P70=0.0006570138001047486, P71=0.0007896772208416311, P72=0.0011377599850358774, P73=0.001553072747715193, P74=0.005389151043228137, P75=0.015136559654910777, P76=0.01540274796047751, P77=0.021545071664245984, P78=0.022473776275912183, P79=0.02331814419833773, P80=0.018760902759755445, P81=0.010470915624328388, P82=0.006626879758341337, P83=0.004577272204923234, P84=0.005076072371630265, P85=0.0049394801994689895, P86=0.0029488338326439005, P87=0.007442967465785239, P88=0.008018088802272316, P89=0.011152167984738524, P90=0.008597245669412588, P91=0.006838620995882051, P92=0.005616967505993139, P93=0.0025132045948491003, P94=0.0033301031343287075, P95=0.0024275661634129294, P96=0.0006815646896865757, P97=0.001888906937561293, P98=0.006440220806586386, P99=0.004423822314173633, P100=0.0014822751648422218, P101=0.0013610985726592857, P102=0.0007276192558454874, P103=0.00020707921122251107, P104=0.00012980153681203365, P105=0.001031790862431234, P106=0.0006922870964527274, P107=0.000328307945873449, P108=0.0002774464513283638, P109=7.479499827349848e-05, P110=0.0001805344269999185, P111=0.0003056175399648895, P112=0.00021825826355921662, P113=0.00017546537764864893, P114=0.0001952673031389087, P115=0.0004638975161055349, P116=0.00030905333627558767, P117=0.00011532038604297409, P118=0.00013001888172935178, P119=0.00023093665624664188, P120=0.0005576921745573722, P121=0.0004064436908278968, P122=0.00019115985060289472, P123=0.0006504613760411394, P124=0.0007961832810602213, P125=0.00013582343721272238, P126=0.0002510255516648498, P127=0.00024834217697131876, P128=9.900191463564826e-05, P129=0.0002092752046117391, P130=0.00013251844259180704, P131=0.00010488610313273176, P132=2.6548210499937994e-05, P133=1.1427276745969056e-05, P134=1.8077950540985153e-05, P135=3.145490358817736e-05, P136=2.2308784268660723e-05, P137=4.552143520766126e-05, P138=3.7802116940784816e-05, P139=5.769466644178595e-05, P140=5.926057240772278e-05, P141=7.658659174157967e-05, P142=3.298271007777831e-05, P143=4.0316792354859274e-05, P144=0.00015799825599313914, P145=0.00032122481964291474, P146=7.55704333944324e-05, P147=4.3744149566553045e-05, P148=6.208353345765999e-05, P149=6.249569783918432e-05, P150=0.00013206227879780283, P151=7.36628465161005e-05, P152=3.5611726091091035e-05, P153=2.3381099016701568e-05, P154=1.4884750565856855e-05, P155=1.6754974938510473e-05, P156=1.711725075589443e-05, P157=1.3476238467614747e-05, P158=2.2575721262781772e-05, P159=4.009293096580957e-05, P160=4.8429700620631525e-05, P161=5.04553224595212e-05, P162=5.3482508510763907e-05, P163=5.826746231001852e-05, P164=7.844974102442102e-05, P165=0.00010369379652842763, P166=0.0001587210640419264, P167=0.000207745038389097, P168=0.00021113877982012808, P169=0.00028696707288758557, P170=0.0002689566153318175, P171=0.00025018885041678624, P172=0.00023309414971053934, P173=0.0003022576157578741, P174=0.0006728840059061018, P175=0.0010142802353565463, P176=0.0002953932995238024, P177=8.930565887783832e-05, P178=2.3132022923928087e-05, P179=1.725452533763419e-06),Row( P0=-728.890388018629, P1=72.40451326036607, P2=8.927002199589307, P3=12.113537524783895, P4=-5.8295545139637435, P5=-2.059428258344852, P6=-5.871851752196082, P7=-3.961769007469328, P8=2.142705104351814, P9=-3.228311644916443, P10=4.667104349234053, P11=3.9689803319506, P12=2.9914095547888886, P13=-3.3546083041504433, P14=2.5688190036159972, P15=0.022367136270406648, P16=2.8864721837328426, P17=9.406171413223097, P18=15.036529632517862, P19=15.41745918492695, P20=11.581938894097048, P21=11.139917852094472, P22=5.990086287495464, P23=1.1976374539206165, P24=-3.6954465187261736, P25=-1.4060548251952985, P26=-0.7066190240762776, P27=1.6263975285028347, P28=-2.4150460372120475, P29=1.0348948266567, P30=-3.1766792291940216, P31=-1.006347623196601, P32=-2.1093820077440197, P33=-0.6651838390204794, P34=-1.5075986314487615, P35=-0.13065462588183516, P36=-2.4619331783587097, P37=2.2206707673243664, P38=2.0570128919085784, P39=3.3923296463912647, P40=0.5437546377443712, P41=0.48761315833571606, P42=0.5603073635585805, P43=0.5622108765223045, P44=0.5536128830680086, P45=0.5719672254988952, P46=0.6398427519025147, P47=0.738818587707654, P48=0.8393195612114582, P49=0.740348677919754, P50=0.6549600452224795, P51=0.5985445653894895, P52=6.035159328672534e-05, P53=1.993341774615517e-05, P54=2.0241619370364273e-05, P55=3.3165277055624463e-06, P56=3.5817517230009055e-06, P57=1.8326526364440777e-05, P58=1.252944780318602e-05, P59=2.1809096547794446e-06, P60=2.74395592630506e-06, P61=1.7270713746553755e-05, P62=0.0003231247603149677, P63=0.0040893959179529775, P64=0.019879035447336133, P65=0.026980713875258768, P66=0.018038343184063384, P67=0.00924207771991002, P68=0.001685485271844581, P69=0.0004297618136526926, P70=0.0002493610750833646, P71=0.0001249570173032454, P72=6.347999479294949e-05, P73=5.218038493562931e-05, P74=0.000101282128369389, P75=0.00030059830909129543, P76=0.0016805135250980358, P77=0.0074445841746516245, P78=0.015115021504744842, P79=0.011258281281122736, P80=0.0034468097964901584, P81=0.002206283713473743, P82=0.0016377102228459874, P83=0.000353016634237081, P84=0.00017926497808195082, P85=0.00014815703957142085, P86=5.967500651144623e-05, P87=0.00016704994336597651, P88=0.0007282475472996456, P89=0.000860038506616378, P90=0.000533853354532116, P91=0.0004282190464782799, P92=0.0006291674447857224, P93=0.0009215102719882007, P94=0.00275319604811521, P95=0.0008928806748899368, P96=0.00014187149438206747, P97=0.00020188767904240465, P98=0.000414389540060883, P99=0.00021784783303712714, P100=8.949310571560769e-05, P101=0.00013101764553267949, P102=0.0009984342505431952, P103=0.0004777001295175113, P104=7.08100409019245e-05, P105=0.00028091972261330846, P106=0.00018298700139359944, P107=2.5297475041891933e-05, P108=6.792153599573558e-05, P109=0.001632960331215901, P110=0.0007971147867033673, P111=6.243744287818107e-05, P112=2.4735516102587925e-05, P113=2.2404313936630216e-05, P114=0.00014016204563668455, P115=0.0004232344359844166, P116=0.00020667547252087634, P117=2.0500894502836907e-05, P118=1.957421885642276e-05, P119=0.00013148616197906834, P120=0.00010026665972527477, P121=4.547442726169874e-05, P122=3.351997675245703e-05, P123=0.00018448161900987755, P124=7.599101201760875e-05, P125=2.160177163965071e-05, P126=2.009056331893142e-05, P127=1.2839457027797755e-05, P128=8.293057702593726e-06, P129=6.884430998601779e-06, P130=3.5471635107610185e-06, P131=7.172088280043638e-06, P132=7.001365195887792e-06, P133=4.208544316069048e-06, P134=7.195152743479643e-06, P135=3.96138237121873e-06, P136=6.324037782778819e-06, P137=9.402741597918688e-06, P138=1.770758548568993e-05, P139=1.5906070733309214e-05, P140=2.5727726582222515e-05, P141=2.3848740546384935e-05, P142=1.4411709667254876e-05, P143=1.3554198226957123e-05, P144=6.554818846822627e-06, P145=1.1211131764708998e-05, P146=1.4350126258544103e-05, P147=2.18344793797564e-05, P148=1.1663165654569824e-05, P149=8.78287371179334e-06, P150=1.4360397138820337e-05, P151=4.6327198906323025e-05, P152=4.8308029881443896e-05, P153=2.157653806526665e-05, P154=2.378988196264011e-05, P155=2.2087879761199987e-05, P156=2.5328169067868472e-05, P157=3.4036164586459944e-05, P158=3.534251796626404e-05, P159=0.0001086903352748416, P160=5.4702204991612524e-05, P161=5.284673803270232e-05, P162=6.970288589671808e-05, P163=4.8025890143213935e-05, P164=2.920947197786175e-05, P165=2.3774756275998253e-05, P166=2.9827686730698746e-05, P167=2.564221954707491e-05, P168=2.7175574168119496e-05, P169=2.120248333911773e-05, P170=1.4130327465537888e-05, P171=4.825060109512791e-06, P172=6.787841186129037e-06, P173=1.2542461333154745e-05, P174=9.309563738572508e-06, P175=4.400533393559005e-06, P176=5.983684095923765e-06, P177=6.320813833243692e-06, P178=2.8797268079407848e-06, P179=4.659659851847028e-07),Row( P0=-552.4777494821184, P1=43.76109148414112, P2=-28.590389149923677, P3=-2.9567028708153904, P4=-10.59624494891715, P5=-16.396334039456953, P6=-7.131543666594541, P7=-2.4416709160828978, P8=8.99828880660687, P9=-5.753957323558291, P10=7.813272143489597, P11=15.482832639245967, P12=8.104170041158232, P13=11.524106250751041, P14=13.549281840219992, P15=11.796287189858598, P16=5.183332354630019, P17=5.591101837596465, P18=6.678986756393645, P19=7.2785383010141596, P20=-0.19194468467293252, P21=5.006255206817264, P22=3.7854622976087824, P23=2.0130015798160907, P24=-3.010106235040079, P25=-1.2973594040444365, P26=0.6641273075344715, P27=3.496111285151527, P28=0.67788815766355, P29=4.05924913166433, P30=2.4318917023791062, P31=2.3417188496665737, P32=1.8288989060738445, P33=1.4213992347495075, P34=-0.5058186125949247, P35=0.923527992873667, P36=0.5061028615033113, P37=3.900328551208707, P38=1.5467478850710907, P39=1.9294454253494087, P40=0.5439711324322759, P41=0.5619813116881319, P42=0.5581336492801343, P43=0.520695097870565, P44=0.47917706912246133, P45=0.4599687983266384, P46=0.47893881951787676, P47=0.5148291541676108, P48=0.6450681032271935, P49=0.6998588511391498, P50=0.6344598488821521, P51=0.5548903591245519, P52=0.006638916444597842, P53=0.0006943784407504751, P54=0.00016848280767767015, P55=4.81174061087371e-05, P56=5.333593844078978e-05, P57=0.0001292968194401179, P58=8.11087564563873e-05, P59=4.2623965848910414e-05, P60=2.650595607890881e-05, P61=2.6399048838754864e-05, P62=0.0001580664584100732, P63=0.0010342324314473298, P64=0.005159492981262868, P65=0.021873986812343815, P66=0.049654769668067526, P67=0.06825342576656468, P68=0.35190306265394006, P69=0.8421787538418917, P70=0.7002212725837419, P71=0.25136952096488474, P72=0.14335386493290225, P73=0.16694421666343698, P74=0.23041759008164556, P75=0.2626962348648581, P76=0.10038810815401368, P77=0.03144293206469135, P78=0.01524573839592499, P79=0.01612311294978147, P80=0.018794213199361736, P81=0.029128161899084157, P82=0.03972725331360669, P83=0.03214422173188438, P84=0.07915534595016205, P85=0.1477792419824645, P86=0.08978063674895509, P87=0.0660366650527144, P88=0.0918337690688453, P89=0.13703069280210767, P90=0.10379246014653741, P91=0.09209355504813137, P92=0.04519424280362394, P93=0.03675225523367416, P94=0.014612439154676868, P95=0.005936651531836301, P96=0.008204367894787868, P97=0.10380566062756759, P98=0.10977588977607067, P99=0.028056910208754043, P100=0.02495111296014052, P101=0.12210558453524195, P102=0.12614976715475018, P103=0.018226276482950284, P104=0.010768999002593661, P105=0.03657348151319982, P106=0.024043869051723673, P107=0.009332939073680284, P108=0.007343984247080138, P109=0.025105923933499197, P110=0.18568481159432862, P111=0.10374512296891852, P112=0.01332967648689738, P113=0.00558013394792749, P114=0.00376793082825261, P115=0.002257813930642967, P116=0.0037353072058912604, P117=0.011682141133044926, P118=0.009856611202001854, P119=0.0064375378284990846, P120=0.010145988544358041, P121=0.0068306320853095, P122=0.0085633297952524, P123=0.011465492808314213, P124=0.01035707646443978, P125=0.004028012963133571, P126=0.0017908758651734014, P127=0.0020589508805530193, P128=0.003684280541406803, P129=0.003957505322934631, P130=0.004397574843415816, P131=0.0008824972143903576, P132=0.0009442358599186861, P133=0.0012428743381582054, P134=0.0013150255700879567, P135=0.0009733902277134042, P136=0.0007025698637288187, P137=0.001208776763484906, P138=0.002133921687568634, P139=0.0011872296248833986, P140=0.0007465450760850595, P141=0.0012836110554631166, P142=0.000890308215460544, P143=0.00037871641304603577, P144=0.0005647428531006323, P145=0.0006233572846563477, P146=0.0007782883691159982, P147=0.0013545552858344248, P148=0.0015405900283076192, P149=0.0009174785400946645, P150=0.0006413885868848265, P151=0.0005267749226992298, P152=0.0005153845568675207, P153=0.00045807811824171026, P154=0.0003879720007411475, P155=0.00024879597512537444, P156=0.00020597038614253454, P157=0.00026137089994694203, P158=0.00041198523398248515, P159=0.0003563979587366697, P160=0.0004051178132213737, P161=0.0003629475110064232, P162=0.0005844149920416018, P163=0.0005788063079863334, P164=0.0005772569478019313, P165=0.00040332398564540153, P166=0.0003631091756519733, P167=0.0004294035667759926, P168=0.00030379990975544805, P169=0.0002605390862384388, P170=0.00041798143154269876, P171=0.0008204160587406128, P172=0.0006652575168686144, P173=0.00039890515636487766, P174=0.0008675065026395855, P175=0.0011511034123862561, P176=0.0014910689621743327, P177=0.001493110859184172, P178=0.0007035189748100298, P179=7.100322301261038e-05),Row( P0=-654.619835832961, P1=51.203551799850416, P2=-12.715529193271598, P3=9.615969836476124, P4=-9.56695305494268, P5=-11.663910351779386, P6=-0.41983993850683177, P7=-0.16283881830474262, P8=3.335141091240703, P9=-3.735811124177287, P10=2.945355975398705, P11=1.7690475059709048, P12=2.3592805200241167, P13=2.588182329996594, P14=3.4889170727583227, P15=5.606680969611525, P16=4.248993378907815, P17=9.140982635525969, P18=10.335662393052424, P19=8.07554157316895, P20=5.147194795777922, P21=7.850006026892245, P22=4.52103073178445, P23=6.890103570512901, P24=1.7437137721341462, P25=0.9800742323801872, P26=-0.7099955493535857, P27=0.2144351094952281, P28=-1.887601197448281, P29=1.7646623318735313, P30=-0.637158577801434, P31=-0.4365748658196267, P32=-1.485045032520559, P33=1.5312352190402463, P34=-1.3810383080427366, P35=-1.1663546298286374, P36=-3.152619349894014, P37=2.638517960991676, P38=0.2932834934599606, P39=-0.06062212723463642, P40=0.6521269327776393, P41=0.6166079548213879, P42=0.5619898826842501, P43=0.5554537039809502, P44=0.6019123970761776, P45=0.648179351922648, P46=0.6828464803189382, P47=0.6833169391192306, P48=0.72768343305989, P49=0.7014406928205125, P50=0.6587607280244818, P51=0.6212145828756467, P52=5.755161372720546e-05, P53=2.876722089610192e-05, P54=2.341498091589735e-05, P55=1.1640794846067593e-05, P56=7.60844760483364e-06, P57=3.1316968393179153e-06, P58=2.3931750478037127e-06, P59=1.346939197852742e-05, P60=9.972703879731611e-05, P61=0.0008087479316066752, P62=0.007290434161644684, P63=0.013331413352964259, P64=0.018248981071856175, P65=0.07475441492622348, P66=0.07163534780870241, P67=0.06632037558858547, P68=0.05118072553807817, P69=0.024385150358363555, P70=0.010918383447735793, P71=0.003779197879565967, P72=0.003410119770449381, P73=0.0035948755584584453, P74=0.007169800170071461, P75=0.00827442830959685, P76=0.00813855085444979, P77=0.007233915480385229, P78=0.010651883981592917, P79=0.033955499528929456, P80=0.04881327064445108, P81=0.027897947887103553, P82=0.02487454781737565, P83=0.038670161429795784, P84=0.07310819025396297, P85=0.04352220115888294, P86=0.011870965406716128, P87=0.011071835591914206, P88=0.01390859335126141, P89=0.00722787264037219, P90=0.00535799494114145, P91=0.00815657135829965, P92=0.006613094708148611, P93=0.003380342615648698, P94=0.0036138882244395073, P95=0.0035210975937369606, P96=0.005297868902186042, P97=0.04557600075383872, P98=0.055246018988127706, P99=0.014597423599693807, P100=0.013839002172366914, P101=0.0029818287878710576, P102=0.0002545198635153077, P103=0.0014861639466183577, P104=0.005924457588753016, P105=0.0030214553747530855, P106=0.0630499067492453, P107=0.045074000693555175, P108=0.006580217451710028, P109=0.0008058792689900318, P110=0.0008200258045743068, P111=0.00035513805804296444, P112=0.0010775938289953146, P113=0.011327678404092309, P114=0.006261043243930774, P115=0.0017954209063821432, P116=0.0003286482640479079, P117=0.0001502628353931364, P118=0.008048463591324101, P119=0.047634074903787745, P120=0.011506457784591584, P121=0.001529216472117765, P122=0.002441423940129157, P123=0.0022128862429436746, P124=0.0021485884125837166, P125=0.002340803886857145, P126=0.0011368248163310846, P127=0.0005770694481928564, P128=0.0018392267639449258, P129=0.00041669013265070915, P130=0.0005191720941856089, P131=0.0007401393147782006, P132=0.00031120186281324895, P133=0.00028264240561896815, P134=0.00029616611642792047, P135=0.000434753598549518, P136=0.00023401038274732875, P137=0.0003611359215707059, P138=0.00040548445574781856, P139=0.0004983797763711698, P140=0.0009608630326964275, P141=0.0007175875893503901, P142=0.0004105658576376489, P143=0.0001241114822453825, P144=0.0001077613265479652, P145=7.406798978813988e-05, P146=4.852514720023138e-05, P147=6.73211936074462e-05, P148=8.099885676828555e-05, P149=8.476784935735083e-05, P150=5.994886984637929e-05, P151=4.732316990759818e-05, P152=3.361329735188252e-05, P153=3.502986105210911e-05, P154=3.805427223398699e-05, P155=2.8011446768679184e-05, P156=2.1374420026824556e-05, P157=2.0548324933988784e-05, P158=2.2653355131216712e-05, P159=2.5175773553698807e-05, P160=2.1377161047339732e-05, P161=2.7523405628119076e-05, P162=5.492220216714299e-05, P163=3.117732308638849e-05, P164=3.765482196755737e-05, P165=6.092638377200492e-05, P166=3.6252101720736365e-05, P167=5.620998928549865e-05, P168=0.00022436268092985737, P169=0.00016176622636837983, P170=0.00010748427587861157, P171=6.753415170508947e-05, P172=6.989487925596324e-05, P173=5.051811145053877e-05, P174=5.088377937252148e-05, P175=9.795148041362564e-05, P176=0.00011534189884851319, P177=0.0001688966893540217, P178=6.355365612924789e-05, P179=5.657234510797299e-06),Row( P0=-717.7792318717517, P1=77.97117725508495, P2=7.00058312883737, P3=12.61333867228344, P4=-5.729185379314833, P5=-3.3965744211652518, P6=-1.988185571177795, P7=-3.790946529402852, P8=4.915302438949427, P9=-2.6564904708389903, P10=2.6980434456563938, P11=3.546436252123999, P12=5.459982061887869, P13=0.025348895265902244, P14=4.923114133277894, P15=1.9274693616419734, P16=4.148666927171166, P17=5.973772752812573, P18=9.299073310050375, P19=10.388710524181233, P20=7.96903296254827, P21=10.232217053170636, P22=6.590489625815287, P23=5.003522192384872, P24=-0.7712673129447232, P25=-0.8672044854654539, P26=-2.040738698320388, P27=1.2981041428026199, P28=-1.3314924144577691, P29=2.567075236803889, P30=-2.838002135906182, P31=0.423689269979041, P32=-0.7232811684238359, P33=1.5952582976848, P34=-0.22526396149106007, P35=-1.0671272148530748, P36=-4.451896756656394, P37=0.11924487662061395, P38=0.2637032563670375, P39=2.666887864999979, P40=0.6589577451180216, P41=0.6080907597829519, P42=0.5695410060928481, P43=0.5897148827549091, P44=0.6320144677431623, P45=0.6472133105421355, P46=0.7140885424717419, P47=0.736061668674696, P48=0.7756926555196219, P49=0.7197164250773593, P50=0.6330948578827397, P51=0.617538571741204, P52=7.399526219974691e-05, P53=4.420895352387199e-05, P54=5.0356685856385706e-05, P55=7.4689335595946595e-06, P56=3.1049654135995195e-06, P57=6.647069808705679e-06, P58=2.195870311517436e-05, P59=9.437741207202064e-05, P60=0.00014706552685984207, P61=0.00028105668422786356, P62=0.0024119160872750844, P63=0.007011207466006515, P64=0.017613674920223706, P65=0.016139996156452705, P66=0.007876963708892487, P67=0.010790735442931998, P68=0.007854662801880576, P69=0.0025793055966562995, P70=0.0007673311329616151, P71=0.0003250428584547688, P72=0.0004337995824985801, P73=0.0007681338788516207, P74=0.0023122187670103224, P75=0.0025305032143381383, P76=0.002562435921886473, P77=0.013369831666921343, P78=0.022097194258051164, P79=0.008726772083337319, P80=0.0010657628119130232, P81=0.0005253099717205008, P82=0.0011702198592585076, P83=0.002971973571290695, P84=0.004890155279295434, P85=0.0043024276479671835, P86=0.001174979704644691, P87=0.0009657652663939185, P88=0.0008713235635085636, P89=0.0007036854916410187, P90=0.0011316538535113484, P91=0.0009798763159289802, P92=0.0003460657463801523, P93=0.00020667287958305762, P94=0.00038177631985478167, P95=0.0005513252922490428, P96=0.0005583897219569916, P97=0.0006818917590982659, P98=0.0007810224225085853, P99=0.001254738257587389, P100=0.0002695056673268457, P101=0.000204808485317666, P102=8.77015977165495e-05, P103=0.0002466639631383559, P104=0.0002902978727797037, P105=0.0017130830707866827, P106=0.0024018678159829876, P107=0.00037643775271747795, P108=0.00013845575783668452, P109=0.00011382869435486265, P110=0.00016018071686569381, P111=0.001030772461174946, P112=0.00037513396479467085, P113=0.0001288536235347223, P114=8.874795529975835e-05, P115=0.0001680250883792418, P116=0.0007086626844839834, P117=0.00011678950779891482, P118=0.00010359768576980104, P119=0.0001303764780557596, P120=0.00036426747779429365, P121=0.00013841135020760024, P122=0.00011521311748515017, P123=7.502989376331947e-05, P124=7.73397764692829e-05, P125=3.496638677049961e-05, P126=2.0837760684857542e-05, P127=1.6769735652801535e-05, P128=5.690079195289801e-06, P129=7.168053634506678e-06, P130=1.1249784512999626e-05, P131=1.1879278533387951e-05, P132=6.149677894387273e-06, P133=7.136950068396585e-06, P134=1.2612144949734333e-05, P135=1.697456201156222e-05, P136=1.8294851562151416e-05, P137=1.0966380701589034e-05, P138=1.0870508320007942e-05, P139=2.0734658647386602e-05, P140=3.1791959614596025e-05, P141=2.8121526533490105e-05, P142=2.7767583540341706e-05, P143=1.684258337329434e-05, P144=1.0569840890402143e-05, P145=1.2574132056972037e-05, P146=9.366843333255071e-06, P147=8.147082257039545e-06, P148=1.1274721748645308e-05, P149=8.573571643985143e-06, P150=1.0235090252924262e-05, P151=5.658458976580346e-06, P152=6.764077594007018e-06, P153=5.961250903307814e-06, P154=6.022099136429862e-06, P155=8.920698523966223e-06, P156=1.3057468264226104e-05, P157=1.0036677382049305e-05, P158=9.343744905217413e-06, P159=1.610737150131242e-05, P160=9.919691886477198e-06, P161=1.255884679588194e-05, P162=1.0354913074034956e-05, P163=2.349237682884237e-06, P164=3.6649529510752447e-06, P165=3.0376344417195785e-06, P166=1.404007704628582e-06, P167=4.435469801544994e-06, P168=1.5517263247361454e-06, P169=1.7623073024408096e-06, P170=3.827381969119661e-06, P171=1.8524448923040167e-06, P172=5.935732559475411e-07, P173=2.1140533283461823e-06, P174=5.948831280673463e-06, P175=2.0793613774194375e-06, P176=1.758727038322992e-06, P177=1.5976103066957167e-06, P178=1.5842453336000661e-06, P179=3.3444146970275105e-07),Row( P0=-740.3271979365846, P1=43.327170896076524, P2=-2.510747360820312, P3=-1.2093655411937045, P4=-23.140934038715944, P5=-10.579915596484943, P6=-9.338609525421715, P7=-3.2242470957579346, P8=-4.918885548150671, P9=-7.5165963832648295, P10=-3.595917189011367, P11=-4.674855610792686, P12=-3.9302258321228, P13=-0.2266980006769811, P14=4.597298983256635, P15=2.220402987490576, P16=3.717807177308921, P17=3.0360901058525, P18=1.7621174405877307, P19=1.9498014726317365, P20=-1.1175002690125762, P21=1.9332380799738547, P22=-3.2062330336667713, P23=-1.653057488799728, P24=-5.8477796245828255, P25=-4.100104820541527, P26=-4.418710317347128, P27=-0.7493742022851269, P28=-1.517605868376001, P29=0.871407664120406, P30=-2.5077861074344203, P31=0.05574902976302767, P32=-2.8884185452518722, P33=-1.4251998153337009, P34=-4.63600227579493, P35=-2.832416588780265, P36=-4.036088562252942, P37=-0.3196504615361185, P38=-1.3062378068895373, P39=0.9898727400053523, P40=0.6406791954294248, P41=0.6575688152106122, P42=0.6223083702102351, P43=0.5541007810967054, P44=0.5236457296365415, P45=0.4814115056381092, P46=0.47512380346881816, P47=0.5016810320705208, P48=0.5414053558375819, P49=0.5437570571647189, P50=0.5583983814895577, P51=0.598344926109933, P52=7.188410623394074e-07, P53=8.840465335904302e-07, P54=5.078418286108324e-06, P55=1.3999876437475462e-05, P56=2.2784342508567563e-05, P57=2.2530411670817274e-05, P58=8.588213610489458e-06, P59=5.3175941625840355e-06, P60=6.307700152085902e-06, P61=2.864102244969705e-05, P62=0.0002128261070341839, P63=0.0008182504618320507, P64=0.0012553344813793495, P65=0.0040404269207391, P66=0.010907637419624453, P67=0.011144785686749053, P68=0.014734562643083822, P69=0.01784767045410626, P70=0.007752824190982056, P71=0.003960368470697931, P72=0.002492909522983388, P73=0.0006768338595730978, P74=0.00020476451510450806, P75=0.0003303591090193273, P76=0.0005303007162550796, P77=0.0011852855686304317, P78=0.001828863591815815, P79=0.0021377599322892978, P80=0.0022890582115661674, P81=0.0021067861580542743, P82=0.0014062587777651406, P83=0.0013031307322859617, P84=0.002337363838101578, P85=0.0033579271510984854, P86=0.0027072296779714375, P87=0.001926423779339049, P88=0.0012904814037194865, P89=0.0014915693343091816, P90=0.002835852589589505, P91=0.0044483860302321696, P92=0.0012044444780107634, P93=0.0002147404806301273, P94=0.00011180585976209396, P95=0.00012190645102584632, P96=4.32461448689464e-05, P97=0.0001932009565700147, P98=0.000588457957629003, P99=0.0006298066637002282, P100=0.00014721582906035186, P101=5.343342182126277e-05, P102=4.323093958127701e-05, P103=5.2966643151615475e-05, P104=6.873713486482444e-05, P105=9.964758684190468e-05, P106=0.0001243231699777777, P107=0.001034320858599175, P108=0.0010460652790608251, P109=7.549405598386608e-05, P110=3.569017560888059e-05, P111=2.260029920329036e-05, P112=3.5313623698020183e-05, P113=2.5343272510504113e-05, P114=0.00013611885935290214, P115=0.00010721703177376138, P116=2.3986027270946972e-05, P117=2.748782125123447e-05, P118=4.921181326192109e-05, P119=2.7232262416194676e-05, P120=2.0979878274273692e-05, P121=1.7993919511783937e-05, P122=1.94389157302059e-05, P123=2.469087502083905e-05, P124=4.102284297711221e-05, P125=4.575789996528159e-05, P126=2.7599523418488027e-05, P127=3.031033521611405e-05, P128=5.4792581275287364e-05, P129=1.538371528536707e-05, P130=1.2093619068088714e-05, P131=1.2713025784925907e-05, P132=1.327190462113133e-05, P133=2.0053170902351156e-05, P134=1.4487081119537117e-05, P135=1.785875797071646e-05, P136=2.459916876043668e-05, P137=2.8875675209790962e-05, P138=2.3554874881222266e-05, P139=1.9074915327786592e-05, P140=2.7739498801158763e-05, P141=3.4381167239165355e-05, P142=3.9401920503696756e-05, P143=4.2466937802011296e-05, P144=5.172632033932343e-05, P145=2.8822849848810224e-05, P146=2.8363225029872638e-05, P147=7.540225005181597e-05, P148=7.782446613902609e-05, P149=6.084523888388324e-05, P150=6.358917024096656e-05, P151=4.002354288307403e-05, P152=0.00010214660206965893, P153=5.626818643712419e-05, P154=3.301114634467959e-05, P155=5.535261847719967e-05, P156=6.696858567741427e-05, P157=4.175369595407312e-05, P158=4.1000062032671765e-05, P159=4.2994369712059424e-05, P160=2.871173363226767e-05, P161=2.7824940882505797e-05, P162=4.79092250573729e-05, P163=5.0601463201853385e-05, P164=4.7922346097999536e-05, P165=3.4574787943562886e-05, P166=2.2055991801725635e-05, P167=1.1585080006008153e-05, P168=1.533420669592637e-05, P169=2.5182113633187397e-05, P170=1.8299913845164606e-05, P171=1.4550850151834718e-05, P172=9.390536246644706e-06, P173=2.346586898374732e-05, P174=2.4012650707693423e-05, P175=2.7164190670104633e-05, P176=1.8785663604587372e-05, P177=2.3976695399070906e-05, P178=1.5812213724825425e-05, P179=1.4807745261642058e-06),Row( P0=-630.1480662536889, P1=19.433302839277747, P2=-20.778050108136668, P3=-10.732321674395429, P4=-21.855290639512557, P5=-16.41300081947931, P6=-10.609213096147966, P7=-11.963217298930358, P8=-7.991154148364496, P9=-9.078141538383106, P10=2.0335648647622935, P11=4.677438926388113, P12=6.811491365360965, P13=7.362977359866057, P14=-1.6138267704757663, P15=-2.795673985869443, P16=-6.878786360630793, P17=-0.19453592197003947, P18=-0.6196662230726732, P19=-0.5484399827098702, P20=-0.9557926654899851, P21=0.22188227577046898, P22=-3.0787210709920405, P23=1.5079565176408782, P24=-3.9409903741084116, P25=-2.6448956980769154, P26=-3.445193206123182, P27=-2.9929875418387213, P28=-2.9733710469916983, P29=-0.28444735096256657, P30=-3.448803376962082, P31=0.0023391039265535105, P32=0.16335821899917988, P33=1.08044572513673, P34=-0.5778880366929827, P35=-0.24191252154271856, P36=-1.7106226258841566, P37=0.8821882656504676, P38=0.014360137095371404, P39=0.9421351543908888, P40=0.441266021059724, P41=0.43173658948343996, P42=0.4532001129768277, P43=0.46630108111192337, P44=0.4950482338278518, P45=0.48807654391847693, P46=0.477411062129401, P47=0.4921157252919476, P48=0.4865560508199717, P49=0.4530653542885157, P50=0.4310182951771272, P51=0.4192071765689961, P52=3.439763183875154e-07, P53=6.499601066839517e-07, P54=1.4802652794184565e-06, P55=4.225504452040366e-06, P56=1.9505095172840448e-05, P57=0.00016670598811705675, P58=0.00024569726916984377, P59=0.0001567676150602826, P60=6.224299760773088e-05, P61=2.4253329922396477e-05, P62=5.4730596002961197e-05, P63=0.00041749632715086543, P64=0.0012366202353256996, P65=0.00281108897823025, P66=0.00639859889588991, P67=0.010229449157645731, P68=0.012132557806526308, P69=0.02157759814255006, P70=0.023067054913182942, P71=0.033199594256186224, P72=0.09944626793510183, P73=0.1577884856827492, P74=0.27224112976986337, P75=0.20494730082203588, P76=0.12841851625023784, P77=0.1139015758624042, P78=0.07427569964336893, P79=0.10035721286017978, P80=0.06165418082157897, P81=0.01883759063210672, P82=0.006178692922910348, P83=0.0036052767872520207, P84=0.003154848097902963, P85=0.002207508438288554, P86=0.002344110993522372, P87=0.0029989070541613255, P88=0.004730502650809262, P89=0.005637237862189215, P90=0.013963110643067074, P91=0.040399658595824575, P92=0.04216769359294055, P93=0.033571260119172094, P94=0.129175218584747, P95=0.17484654119486434, P96=0.07305409071705714, P97=0.11130039063083377, P98=0.0924041781341133, P99=0.02027378302946062, P100=0.013349579353851842, P101=0.0062119302770266005, P102=0.0018609922008374163, P103=0.0017896504558186518, P104=0.0037361504714965355, P105=0.008609759346786039, P106=0.008101300233474027, P107=0.008921929817435647, P108=0.014581364427598184, P109=0.033559367081224874, P110=0.029060089602782828, P111=0.014939760785292367, P112=0.011693321706567304, P113=0.002956924016191043, P114=0.0028753176856411173, P115=0.009677209844318883, P116=0.008310999508173633, P117=0.0034053017085554, P118=0.0034310331524293925, P119=0.002696485597365664, P120=0.0017111324872596315, P121=0.0020229142113814894, P122=0.0024801615014219108, P123=0.002608389207854405, P124=0.0011854475767547506, P125=0.0021332063980700495, P126=0.0012374720326870142, P127=0.0016553019687380011, P128=0.0009459144075030125, P129=0.0004859983692941026, P130=0.0007281702655354997, P131=0.0010269899582230895, P132=0.0010866673300681797, P133=0.002494884781506381, P134=0.001998815042922295, P135=0.0005189416826945642, P136=0.0004110021184192355, P137=0.0004978866818388057, P138=0.0007679793882582972, P139=0.000964547991611981, P140=0.0009671353042002388, P141=0.0007770623346482776, P142=0.0008733734384069063, P143=0.000991855527640019, P144=0.00036182576655330854, P145=0.00018125138752566914, P146=0.00020111961894322684, P147=0.00020399250498164646, P148=0.00034037956177646863, P149=0.00042684497639844415, P150=0.00036386208345411017, P151=0.00027275840608952437, P152=0.0002868928685656381, P153=0.00019060018640223886, P154=0.00012576266004160215, P155=0.00016102482812812083, P156=0.0001449150959613124, P157=0.0002271119386276326, P158=0.0003887971666919015, P159=0.0002493185458777626, P160=0.00015818631946616353, P161=0.0001617162606419267, P162=0.00025198114365647596, P163=0.00034538472271188926, P164=0.00038608898811119625, P165=0.0007727843797288258, P166=0.0005635767912602945, P167=0.0006075157092060138, P168=0.0004963469911928996, P169=0.000548415343967181, P170=0.00045479178025327314, P171=0.0002762781271414564, P172=0.00036730577409262383, P173=0.0005242037777452746, P174=0.00027944377531087544, P175=0.0002854296864094559, P176=0.00017721901301911438, P177=9.94158208445236e-05, P178=3.382218198947597e-05, P179=4.559206941499533e-06),Row( P0=-579.4002080512408, P1=22.144120477112526, P2=-23.16398153376087, P3=-8.39297253891742, P4=-17.49748913124423, P5=-11.516747752396954, P6=-5.193910315045901, P7=-8.203668617602977, P8=-0.1378820463507531, P9=0.7686965647958081, P10=12.51664942166028, P11=10.936149864408666, P12=6.624068481147636, P13=3.1678725300209063, P14=-4.196420601484054, P15=2.5214320091620537, P16=-1.6188502606457853, P17=5.0686278109157445, P18=0.6037356943752372, P19=0.6571440884853499, P20=1.016639870026236, P21=2.1608643089284696, P22=-3.3434266481834847, P23=-2.7167866099287723, P24=-2.489947782244789, P25=1.2528608266665442, P26=1.0530674413561454, P27=-2.5327560384135315, P28=-2.1003563175815643, P29=1.247110921776748, P30=-1.4596612954641952, P31=1.1268331100192652, P32=-0.5164857087824121, P33=1.0877675362256178, P34=0.0544602820961608, P35=1.4967913855955137, P36=0.09127033739132388, P37=1.0361201203737695, P38=-1.899813118519588, P39=-0.6065703873934185, P40=0.460417421778241, P41=0.44402202440439426, P42=0.47594622498026573, P43=0.47381643007857005, P44=0.4998447403885399, P45=0.5304852018052257, P46=0.5298428001751154, P47=0.52963840438449, P48=0.5110425682009605, P49=0.5331783250752608, P50=0.5499983297131256, P51=0.5287843530173196, P52=0.0009781333892260303, P53=0.0009600635405360689, P54=0.0005430153177014403, P55=0.0004962750047337606, P56=0.00043866598892619227, P57=0.00022044948903857215, P58=0.00014364169567880971, P59=7.110480456887938e-05, P60=3.9307565792672684e-05, P61=3.5126283816582414e-05, P62=4.7120063832900353e-05, P63=4.290859247406656e-05, P64=5.5476015291019206e-05, P65=0.0007902677649774257, P66=0.005386523937849328, P67=0.009272935314503022, P68=0.008520424642109455, P69=0.017881196355631755, P70=0.023493297819118915, P71=0.06667820593388313, P72=0.26211253859510475, P73=0.38588670557257365, P74=0.18246155790394136, P75=0.1483520542399894, P76=0.11008186543488331, P77=0.1565381417983898, P78=0.2997181702784864, P79=0.5069326087577862, P80=0.4823858792070119, P81=0.2830691694256997, P82=0.16275012670936587, P83=0.043208998773456415, P84=0.008686634848225376, P85=0.00453829052930286, P86=0.0033507890926736746, P87=0.0037376608982647584, P88=0.0039934296742482394, P89=0.004803540713337284, P90=0.011924736842138446, P91=0.026309729155150577, P92=0.02732447630234518, P93=0.017887138912404784, P94=0.03134318602198602, P95=0.07466682443256124, P96=0.06373012722877386, P97=0.045810918312608945, P98=0.03969094544169779, P99=0.045453678986661784, P100=0.18987617328687284, P101=0.24699558909721078, P102=0.23294926653203282, P103=0.04118233428682277, P104=0.010266626106662098, P105=0.004837812019067728, P106=0.0039430863287816765, P107=0.005526348322704299, P108=0.009558858613003741, P109=0.011986273765205372, P110=0.02209726916848567, P111=0.02362604848894489, P112=0.024147066841253437, P113=0.032583981711689754, P114=0.05633330886574671, P115=0.08307851931232745, P116=0.010275796635776982, P117=0.0034320106263591565, P118=0.0031197002544141746, P119=0.0024511155888360904, P120=0.003038672217110538, P121=0.005852141372551571, P122=0.003669937442749285, P123=0.0035219608507601444, P124=0.006448138725044611, P125=0.005476894216458893, P126=0.003025223436443298, P127=0.004014310157580799, P128=0.0030973499367179936, P129=0.001515393278670392, P130=0.0027291327562383037, P131=0.007128281379052356, P132=0.005906298120794091, P133=0.0032108456786314726, P134=0.006383935985097361, P135=0.0032543949477483107, P136=0.0033842101315144178, P137=0.0037961934735361883, P138=0.0026339469670648833, P139=0.0032659474106956983, P140=0.004382397079106372, P141=0.004207910213374984, P142=0.0020210366833469284, P143=0.0012689599791284557, P144=0.0009465515980361443, P145=0.0008191974202474581, P146=0.0008595272348609308, P147=0.000529119957653403, P148=0.0007168289769308974, P149=0.0009510258346477449, P150=0.0009569840740522067, P151=0.0009341708429934193, P152=0.0008063980295741975, P153=0.0006538332770163835, P154=0.00043288936089331154, P155=0.00037201794699972873, P156=0.00025687669262646146, P157=0.00037034950814959013, P158=0.0005580731158270145, P159=0.0007694461834072034, P160=0.0004143743309478207, P161=0.00022213806742049506, P162=0.0002643046648894567, P163=0.00021722093866412766, P164=0.00018978940044380533, P165=0.00018988794353016255, P166=0.00020046174643877755, P167=0.00020621917500681207, P168=0.00020086220620671913, P169=0.00023002790443170908, P170=0.00015089388138392814, P171=0.00010831258907826725, P172=0.0001645711783582266, P173=0.00012471852134374875, P174=0.0001452404902561673, P175=0.00017097120790264626, P176=0.00014414142413784873, P177=0.00011653804203758044, P178=5.5671149600970815e-05, P179=6.817533045118001e-06),Row( P0=-513.2305117605774, P1=-2.7158674409568477, P2=-40.212985214846064, P3=-8.833770369791358, P4=-25.221131702199077, P5=-14.049262746204128, P6=6.104231251065311, P7=1.6481359807106304, P8=15.429538797846194, P9=1.1511946552744894, P10=1.9014539455640673, P11=-4.830398169685801, P12=-6.908273925054252, P13=6.222647822715267, P14=0.946064973392742, P15=-0.13350246010501007, P16=-9.103689538807854, P17=0.9375404140122248, P18=1.0597868859376156, P19=-0.18498099896092235, P20=0.8797699512790571, P21=3.9969456581855325, P22=-1.4169589355888559, P23=1.3242435091146976, P24=-2.8325179760984542, P25=-1.506243654116764, P26=0.06507765871123995, P27=-1.4932655146611955, P28=-0.9978138115009241, P29=0.7936412952460429, P30=-1.7693081474951784, P31=0.9382418656100131, P32=-2.1950394654724144, P33=-1.076809768147554, P34=-2.382435102798946, P35=-0.5981328226288246, P36=-1.0582940336753817, P37=-1.3597710538680334, P38=-0.6645422435071919, P39=0.7780817620330985, P40=0.5481752573561862, P41=0.5787372795247154, P42=0.5644476495228951, P43=0.5431153949175739, P44=0.48503890627114077, P45=0.4375896643819995, P46=0.4093667411985291, P47=0.4271042936570929, P48=0.43877942223085875, P49=0.46425995516692115, P50=0.49621925602505923, P51=0.5018223134024775, P52=5.512027931343437e-06, P53=1.858325503225071e-05, P54=5.5315310695391264e-05, P55=7.670665947145216e-05, P56=8.026811578806217e-05, P57=7.442646198535421e-05, P58=4.2316674562270394e-05, P59=3.3027241531340996e-05, P60=6.905815294275204e-05, P61=0.00013219408416672134, P62=0.00023046132184017701, P63=0.00017230712997880112, P64=0.00019907156064763448, P65=0.000459328349378807, P66=0.0003961629546859372, P67=0.00025789340111587313, P68=0.00032888590281626837, P69=0.0006495248324118049, P70=0.0007761034697238849, P71=0.001389597538084563, P72=0.0032187846204868027, P73=0.005750051307379106, P74=0.019880903656857154, P75=0.037955427514744663, P76=0.042264205790764564, P77=0.08797104453381957, P78=0.16614843084559555, P79=0.19690973462975436, P80=0.2143243639696505, P81=0.1636357717784202, P82=0.23824512155636268, P83=0.45793219522215955, P84=0.7701980893726482, P85=1.4678400316851241, P86=1.8247557080185295, P87=1.7118235069077257, P88=1.4577634393185823, P89=1.1512149631893513, P90=1.1673856626824102, P91=1.0817246141505965, P92=0.4758232105577448, P93=0.5411732601545621, P94=1.464009398469388, P95=0.6553636295785535, P96=0.024829326032509678, P97=0.030435277442619663, P98=0.03141526469641272, P99=0.03342389248263459, P100=0.060032364648552924, P101=0.05969579541029908, P102=0.1148492721552281, P103=0.1479232497269104, P104=0.2809756721654338, P105=0.8339121696201244, P106=0.5281509648028164, P107=0.22874320760173966, P108=0.13986263618110342, P109=0.20122830284904378, P110=0.5117828490534589, P111=0.7978207134148096, P112=0.6515364345501329, P113=0.574716666779979, P114=0.43462902316575225, P115=0.2464177010370092, P116=0.11482393204538449, P117=0.11714667046990898, P118=0.16357344448595795, P119=0.09208673691126626, P120=0.02284611128167793, P121=0.09867664200991749, P122=0.32583209579114547, P123=0.17500081114207794, P124=0.10789611745564913, P125=0.10719810199409704, P126=0.057577338983011345, P127=0.061758861621468715, P128=0.049758676795450936, P129=0.026390856123442916, P130=0.0460336067080982, P131=0.032644101597719535, P132=0.04639760659398969, P133=0.018222690492320465, P134=0.00947817735089741, P135=0.014879473685048723, P136=0.013054048105256478, P137=0.023797092005182024, P138=0.03201184371381612, P139=0.04986132665848579, P140=0.029046117459920064, P141=0.03209465852464709, P142=0.03431812462587694, P143=0.03778859336613897, P144=0.021995866248482022, P145=0.01021889387882397, P146=0.006594833350638237, P147=0.004459228412182669, P148=0.011643923839110332, P149=0.025967746909213198, P150=0.025181836541257906, P151=0.007600357607213077, P152=0.005202068534421705, P153=0.004509079612802722, P154=0.0025073897990875865, P155=0.000821245673726186, P156=0.0006563717663836616, P157=0.0005645328013679433, P158=0.0005569646455032127, P159=0.0005225711207874414, P160=0.000466974676003861, P161=0.00026666330006468923, P162=0.00041972791072393897, P163=0.0005295772005947514, P164=0.0008116214932100486, P165=0.0005933428856244141, P166=0.00069819605220495, P167=0.00040014766757784836, P168=0.00049071344199908, P169=0.00048275933088615737, P170=0.0005569029152234248, P171=0.0005956620262250406, P172=0.0006714290877153332, P173=0.0007269923322486361, P174=0.0008292188420067474, P175=0.000977644412266177, P176=0.0006406021063934049, P177=0.0006676698590340926, P178=0.0005689585993536982, P179=8.423836651059258e-05),Row( P0=-767.8970648170974, P1=35.73900709591482, P2=-6.632207570020148, P3=-10.05237931919512, P4=-18.27959069389041, P5=-7.8620949735937256, P6=-9.472820175174368, P7=-5.736336989685612, P8=-7.953602697908383, P9=-12.024485519645172, P10=-4.506408592319588, P11=-6.906175265598159, P12=-1.6795304034157745, P13=-1.2298066063992306, P14=2.7281662222399072, P15=3.969758198286582, P16=3.6180398057016054, P17=6.596496481284997, P18=4.292114519670388, P19=3.6425810549800834, P20=3.5005721444144693, P21=1.4899187247097607, P22=-0.22214379811641455, P23=1.4423795193606153, P24=-3.6962575887829083, P25=-3.9045368801656903, P26=-5.890732119283785, P27=-4.223781402384247, P28=-2.726324540694341, P29=0.2618519286865573, P30=-3.6722810376913047, P31=-0.18830087160706832, P32=-2.221747348277268, P33=-1.9204089722530002, P34=-3.8960328669823654, P35=-2.7003362067312917, P36=-2.1200336081056124, P37=-0.2640068035229232, P38=-2.1676016819385295, P39=-1.5420437733993286, P40=0.4862466314578091, P41=0.5180341697572785, P42=0.48989799963323816, P43=0.4273966020978632, P44=0.4231857519079066, P45=0.42078470076501173, P46=0.4345988314439628, P47=0.4631200417021675, P48=0.4612179210265805, P49=0.4547028853128939, P50=0.4983103017051193, P51=0.49775077366446163, P52=1.3004002598904835e-07, P53=3.5156073741467057e-07, P54=1.8858808354525172e-06, P55=2.684505244352798e-06, P56=4.940573547404186e-06, P57=4.134691070961639e-06, P58=1.5094789412362668e-06, P59=7.23769062687129e-07, P60=1.1816686357617932e-06, P61=5.389129437469857e-06, P62=0.0004278032543918844, P63=0.0022283648848742178, P64=0.0020397905151490927, P65=0.008343461841513695, P66=0.017684749018697484, P67=0.007516988847212618, P68=0.011553727195761712, P69=0.0117069980174181, P70=0.0035297706456376365, P71=0.0005817252287102309, P72=0.0001412437291749163, P73=7.778655667746739e-05, P74=0.00013489367087824607, P75=0.00043196377880550495, P76=0.0005158736873705602, P77=0.0006556300619457495, P78=0.0005662625213199036, P79=0.0004831156034350862, P80=0.0008242096668447346, P81=0.0009468292648297615, P82=0.0007733762682044608, P83=0.000755945134308337, P84=0.001460201928870821, P85=0.002821677694181745, P86=0.0039006676236289627, P87=0.0042960230942912505, P88=0.0024888561405083855, P89=0.0011909010916543493, P90=0.000718072937920174, P91=0.0004472936963128824, P92=0.0003099858446795995, P93=0.00043834500140209413, P94=0.0005911027046067971, P95=0.0004228774652554096, P96=0.00020807283044205929, P97=0.0008892653706511754, P98=0.0024636668156806105, P99=0.0004893717748007221, P100=4.57965678879335e-05, P101=1.949691849589474e-05, P102=0.00016499227873096513, P103=0.00019606506551000228, P104=2.685928043135774e-05, P105=1.2517890423154708e-05, P106=0.0006370648449983904, P107=0.0015430853541782232, P108=8.387057622247564e-05, P109=1.868600141169065e-05, P110=1.0510577823168237e-05, P111=4.082811035558135e-06, P112=5.217516832959639e-06, P113=9.427026786437494e-05, P114=9.422416134377552e-05, P115=9.329750825209833e-06, P116=8.086886632125676e-06, P117=9.784551570876112e-06, P118=1.0486157472872682e-05, P119=5.6513550751720935e-05, P120=2.339116471419698e-05, P121=1.1575728629739018e-05, P122=6.703204825776246e-06, P123=6.079414863416592e-06, P124=1.8043018409195446e-05, P125=1.6684160766011314e-05, P126=9.462309296398487e-06, P127=1.0378414195643098e-05, P128=1.6828041983699912e-05, P129=9.406652089008065e-06, P130=9.33735056207218e-06, P131=9.605837555811317e-06, P132=5.8804645532782845e-06, P133=7.444591274717336e-06, P134=4.993909491051661e-06, P135=3.6937819020687358e-06, P136=4.91084877135401e-06, P137=3.571197464918067e-06, P138=4.034139460829933e-06, P139=6.134825967838131e-06, P140=6.62122541382054e-06, P141=8.329739742907242e-06, P142=7.923466038945682e-06, P143=8.524007662930722e-06, P144=5.638461103001377e-06, P145=4.231388820183552e-06, P146=3.5547551104702573e-06, P147=4.397926241398733e-06, P148=9.140040240003635e-06, P149=1.312622230947953e-05, P150=9.897363083012285e-06, P151=2.388553749292431e-05, P152=3.1663332427889535e-05, P153=1.7253989299842726e-05, P154=1.728517941831074e-05, P155=1.9054469039660932e-05, P156=2.5502490750689463e-05, P157=3.411210120323097e-05, P158=4.41255351879792e-05, P159=5.987827687539904e-05, P160=5.741073103267756e-05, P161=5.198119188871646e-05, P162=5.891392760844957e-05, P163=5.9961080864768926e-05, P164=8.11518507153513e-05, P165=6.800908697035749e-05, P166=6.095815286219498e-05, P167=4.498840461379912e-05, P168=3.2588389353473094e-05, P169=3.0062864111452607e-05, P170=3.017031546571204e-05, P171=4.399332573450948e-05, P172=4.097679109956594e-05, P173=6.698684088321648e-05, P174=3.729277027699307e-05, P175=2.697556485736425e-05, P176=6.383391040727349e-05, P177=6.398202224893852e-05, P178=2.393730815228538e-05, P179=2.998113956475876e-06),Row( P0=-597.3674880039634, P1=33.290073604135806, P2=-19.45712289304513, P3=-17.810568296140794, P4=-26.454620999191125, P5=-11.589874900965075, P6=-12.903142035874172, P7=-7.8857593915457835, P8=-6.190879730086241, P9=-11.01220389477421, P10=-3.902109549711114, P11=-1.472227233084453, P12=4.308406617005756, P13=9.820212117050467, P14=6.034117280678915, P15=4.7485714267323775, P16=-1.4221860975786391, P17=-2.627421664097478, P18=-2.533964829946827, P19=-2.9439924169965472, P20=-1.2690364244932477, P21=1.199193405715679, P22=-1.7420933786920487, P23=0.015411397331208929, P24=-3.920894099283939, P25=-0.5183020585081466, P26=-1.8456181168316803, P27=-0.013193033071851993, P28=-1.589066715423965, P29=-0.20398689357308142, P30=-0.875384296151355, P31=-1.3344364820516088, P32=-1.885874682436981, P33=-0.27969468926815133, P34=-2.693761678911632, P35=-2.1811430991640144, P36=-2.6321289576270495, P37=-0.43389021068204303, P38=-1.0708351023792644, P39=0.7584018782048703, P40=0.5510652569844291, P41=0.6425478013787143, P42=0.676478068410928, P43=0.6650147692504887, P44=0.6899122207233761, P45=0.5913661280882435, P46=0.5345463421569348, P47=0.520707064851692, P48=0.4934141613220926, P49=0.4939252798730161, P50=0.4852103463286597, P51=0.4929635027690082, P52=2.7221767009221247e-06, P53=1.0398295231605867e-05, P54=3.475658975237373e-05, P55=3.743378548253538e-05, P56=2.6611787865743498e-05, P57=2.0335668334140767e-05, P58=1.3756650812044257e-05, P59=1.4518407250614256e-05, P60=2.9682490975990114e-05, P61=7.466507736451814e-05, P62=0.00026508375373119995, P63=0.0009049887817371314, P64=0.0011285539789346846, P65=0.0024518228024793674, P66=0.005759031915222886, P67=0.009103850593967266, P68=0.024920826281491594, P69=0.05093575073781527, P70=0.05212833398146887, P71=0.06963427981582299, P72=0.12130979652903674, P73=0.14060579900127707, P74=0.20234663503216993, P75=0.13715768732204095, P76=0.10140185645842485, P77=0.053053012363681706, P78=0.011784131089942267, P79=0.002965993107271608, P80=0.0013949774342034902, P81=0.00136100931410853, P82=0.0017704577962321028, P83=0.0034386535456903656, P84=0.03982815904463797, P85=0.18524436032498653, P86=0.15648186026585803, P87=0.09602808272389478, P88=0.07432020179490068, P89=0.07502397112688522, P90=0.1269058502099154, P91=0.3637767072706941, P92=0.15620868741880678, P93=0.017090873012564664, P94=0.011443589370623085, P95=0.007596110177667644, P96=0.00832532545146574, P97=0.016571693206086986, P98=0.02364581311891114, P99=0.02994810497438029, P100=0.028192099349480023, P101=0.020477700978191054, P102=0.02521083892911787, P103=0.016394966309365287, P104=0.007288428769097935, P105=0.0030684975861527417, P106=0.002669575079817308, P107=0.010239795260110755, P108=0.01792886464868973, P109=0.015254399915842019, P110=0.017181563760832314, P111=0.014436057291137137, P112=0.009852697883326727, P113=0.005129583007478777, P114=0.009630768573248598, P115=0.00829419988572926, P116=0.0037184851459397014, P117=0.00321856839005006, P118=0.003007579602986728, P119=0.00144698598102145, P120=0.000865454552909294, P121=0.0007396163450452621, P122=0.0006348661264363275, P123=0.0004485166396131716, P124=0.0006618305641257639, P125=0.0003918676823411057, P126=0.0003996398358008042, P127=0.0003969213969534812, P128=0.0001846485161492088, P129=0.0001349555436800459, P130=0.00011534668070189726, P131=0.00021188432691065655, P132=0.00022786652605868074, P133=0.00029858729035152164, P134=0.00024562956870080566, P135=0.0002594830105150816, P136=0.0002588013489213268, P137=0.0001982670679882612, P138=0.00023865879413484772, P139=0.00016117181619945402, P140=0.0002462614572274058, P141=0.00021767331969380825, P142=0.00023542626776693054, P143=0.00017027577675537406, P144=0.00016381118375692212, P145=0.00011809047901339788, P146=0.00011998140135215477, P147=0.00011797058544656935, P148=0.00012557003070756838, P149=0.00015232706345625737, P150=0.0002290454460260077, P151=0.0002761246796923387, P152=0.00019928598575203135, P153=0.00021761348569654916, P154=0.00028504053061804226, P155=0.00031891715123556476, P156=0.00023902227610998674, P157=0.00014941477304077945, P158=0.00013199142740489688, P159=0.00025166365778470106, P160=0.00041528741962545593, P161=0.00035829975192465223, P162=0.0003561168834482406, P163=0.00045913108817248395, P164=0.0007945756605164583, P165=0.0006288302642312418, P166=0.00039624565141973606, P167=0.00042943941514335464, P168=0.0004853772368247793, P169=0.00038609775416833706, P170=0.0003617519175091295, P171=0.0004930227497037826, P172=0.0003743683144689668, P173=0.0006368593621544449, P174=0.00048041415535840325, P175=0.00034344594523792054, P176=0.0004490762176761555, P177=0.0005188295102986541, P178=0.0002520408403805179, P179=2.1155407521999497e-05),Row( P0=-738.3204068778417, P1=35.282031803874915, P2=-13.115555428204575, P3=-9.827862153543578, P4=-20.685276060379685, P5=-11.048972933277721, P6=-7.985859413425128, P7=-7.568510492691781, P8=-6.95566046979434, P9=-5.893759721734212, P10=-1.2828293519906204, P11=-1.5380231038826238, P12=-1.575530081150312, P13=-5.605623094648825, P14=-5.348087708227162, P15=-5.416352279424056, P16=-5.786503330195252, P17=0.3307954504828847, P18=3.2510345316640374, P19=3.2942941554687803, P20=6.2512123626728435, P21=4.1836612527109365, P22=1.8693064560860662, P23=2.805138193061932, P24=-3.7979087346589426, P25=-1.6080258564811365, P26=-2.2249831349786113, P27=-1.286403014851277, P28=-3.7731086115807737, P29=-2.0461229531072744, P30=-4.51318470861932, P31=-0.6513298855274208, P32=-2.069751158366301, P33=-1.6425141463721316, P34=-5.353119494542129, P35=-4.555357113337551, P36=-2.1278691285325833, P37=2.686618254177743, P38=0.8218397964108224, P39=-0.0020198557505492877, P40=0.3890164829310172, P41=0.4536540711859484, P42=0.4109114430352327, P43=0.393244580949277, P44=0.4444557868577126, P45=0.4777674879838548, P46=0.552743099148239, P47=0.5289674061714568, P48=0.5430393019031176, P49=0.5945980292042982, P50=0.4947865389723932, P51=0.40626020464620904, P52=7.507139616070848e-08, P53=4.916393348260468e-08, P54=3.3031855034182315e-07, P55=1.611018324983315e-06, P56=4.599571416072478e-06, P57=3.650543671480877e-06, P58=1.5331765501189324e-06, P59=1.271126278140032e-06, P60=3.30873932943211e-06, P61=4.16190385940167e-05, P62=0.001424016090019693, P63=0.0024446067144330216, P64=0.0072278475913714925, P65=0.010259799363470002, P66=0.0022070350005114416, P67=0.0006038006435807715, P68=0.00016509257736737666, P69=8.828639971715694e-05, P70=0.00010170563705785862, P71=0.00017677495804750367, P72=0.0004557128519595686, P73=0.0016309643462911218, P74=0.008525885761714183, P75=0.00686712803813551, P76=0.0060459319988857155, P77=0.023813762445455126, P78=0.045744146518973405, P79=0.0359435777015776, P80=0.005482218856265497, P81=0.000662815186147597, P82=0.00042288718672749023, P83=0.0001818187629659615, P84=0.00016744566941820333, P85=0.0002796475646485117, P86=0.00015512697335317842, P87=5.637717524358654e-05, P88=4.746028819650267e-05, P89=0.00014537895939383504, P90=0.0005223077153724417, P91=0.0007325565384410643, P92=0.00030535213707705923, P93=0.0018109029429138968, P94=0.006829663886050769, P95=0.004388349528109865, P96=0.0006337298665712392, P97=0.00011916009664977968, P98=0.0006901630082508231, P99=0.0017376012806334209, P100=0.0024404347166880748, P101=0.000553095310989035, P102=3.022804045785417e-05, P103=1.9871893883675047e-05, P104=1.7144723685954357e-05, P105=0.00016134501720730572, P106=0.0005931841770824459, P107=0.00324563776823776, P108=0.0005011382350749413, P109=1.2824769586913972e-05, P110=1.940161254241934e-05, P111=4.665300197478894e-05, P112=0.00019471037827131172, P113=0.000262495668730949, P114=1.0112879714136256e-05, P115=3.9892742379213e-05, P116=4.6993167161703095e-05, P117=0.00020286629526098234, P118=9.94951552831783e-05, P119=1.2040107245156313e-05, P120=2.6616176711454118e-05, P121=3.374737957378345e-05, P122=0.00012976933775136413, P123=9.316783749926811e-05, P124=4.108484858918134e-05, P125=7.842828578592069e-06, P126=7.714724821063623e-06, P127=0.00010299801782476185, P128=0.0003395392661593622, P129=3.3744327504541764e-05, P130=8.856041385242128e-06, P131=1.3523118632238064e-05, P132=1.9355304931993366e-05, P133=2.333782269027216e-05, P134=1.229593385256721e-05, P135=3.7251999469455826e-06, P136=1.0901242929120828e-05, P137=2.254159841163344e-05, P138=3.454580723522849e-05, P139=1.6104629560258594e-05, P140=1.5278737139524112e-05, P141=3.0471142729955933e-05, P142=2.979406547328142e-05, P143=1.582229583525098e-05, P144=1.2399047927559183e-05, P145=7.75907539717587e-06, P146=8.577382487025374e-06, P147=3.3219459445137345e-06, P148=3.79948075165682e-06, P149=9.055138392782625e-06, P150=4.477207376739128e-06, P151=3.9870596427421925e-06, P152=9.161139766676384e-06, P153=5.303416536737983e-06, P154=6.265765634797033e-06, P155=6.557550185642261e-06, P156=4.946763068829018e-06, P157=3.5216275042247396e-06, P158=3.857089112282773e-06, P159=5.877549282161514e-06, P160=6.752210189092399e-06, P161=1.4298583521922684e-05, P162=1.3607490525016827e-05, P163=3.982960777995139e-06, P164=2.4956101404359867e-06, P165=6.310622480536758e-06, P166=6.4735029104068525e-06, P167=3.5184570433466215e-06, P168=6.075134738785032e-06, P169=6.593084077301593e-06, P170=7.286328895936671e-06, P171=5.1067426500509805e-06, P172=7.305092117393589e-06, P173=9.790855095236761e-06, P174=8.259377761236505e-06, P175=4.573677809772482e-06, P176=5.024434508766052e-06, P177=4.286812097681372e-06, P178=2.9557019095902295e-06, P179=5.743498490487406e-07),Row( P0=-691.840062741686, P1=32.90452773309465, P2=-11.610366025468567, P3=-2.30761126198426, P4=-16.969494615802915, P5=-4.873399589885883, P6=-8.322278756075805, P7=-2.8313861181169924, P8=-5.012748747717205, P9=-9.513446033886076, P10=-1.980376820250123, P11=-1.9662683183404075, P12=-0.12048657241390577, P13=3.6217054796257444, P14=5.238249741966378, P15=2.297984982246265, P16=0.7891546558062963, P17=0.2810182352392133, P18=0.6487728278765542, P19=0.43105024837085393, P20=0.8588406154109772, P21=1.6557711394253216, P22=-3.40673652229801, P23=-2.5518928927676674, P24=-5.560119525073943, P25=-0.9709043457578377, P26=-1.319359256836726, P27=0.24414818226407223, P28=-0.9438495377893159, P29=2.763740169819349, P30=-1.6343105103878928, P31=0.9910474048723726, P32=-2.276322155936777, P33=-0.43489605632651285, P34=-3.5272632343230015, P35=-2.6211514471558535, P36=-1.4758718627002796, P37=0.2589182217391362, P38=-1.4453197975181975, P39=1.4821729602043494, P40=0.7157256940933984, P41=0.7747046862645441, P42=0.7543986142445835, P43=0.6821463369916858, P44=0.6733507454578045, P45=0.605458037135388, P46=0.5775768891798754, P47=0.579204401231426, P48=0.6270653716592158, P49=0.67602336868156, P50=0.6913244542236032, P51=0.6762588254283949, P52=2.595826629877311e-05, P53=1.9775933986608722e-05, P54=1.9538887053517803e-05, P55=2.946324180633515e-05, P56=4.2760150713082575e-05, P57=4.9296687110098545e-05, P58=2.8803766399109084e-05, P59=2.024415611568323e-05, P60=1.3537247274350287e-05, P61=2.2034686092314627e-05, P62=0.000149296429503856, P63=0.0006120617955474372, P64=0.001122138510991876, P65=0.0021171525563689012, P66=0.003338476739310596, P67=0.004592481184053314, P68=0.01882482210623349, P69=0.045652588359422326, P70=0.029806034474950522, P71=0.009979853981123447, P72=0.01012105340501316, P73=0.004255944162893784, P74=0.0005113587754679687, P75=0.0001801064085531574, P76=0.00010875706573187406, P77=0.00015779795270788447, P78=0.0002442952825162433, P79=0.00025172587114176827, P80=0.0001680263672231885, P81=0.0001489569781490018, P82=0.00017764867922915272, P83=0.00041384595687295666, P84=0.0016435439403488937, P85=0.0034789979939246463, P86=0.0032902438919672505, P87=0.0028168214139355006, P88=0.0015699698839918127, P89=0.0011302593552296517, P90=0.0022441142076120033, P91=0.006711339451165832, P92=0.002874520558700433, P93=0.00043349470965634344, P94=0.00029942595402188523, P95=0.0004138216739054221, P96=0.0009458066450624716, P97=0.001870919206393232, P98=0.0010409964283433907, P99=0.001041871757386638, P100=0.001205466398595275, P101=0.0010190504774039268, P102=0.0005136140987032962, P103=0.00047774976315194807, P104=0.0010637683940107714, P105=0.000809692328815862, P106=0.0005005679646118223, P107=0.0004799499227461387, P108=0.0003161142078907502, P109=0.00011447083568340519, P110=0.0001272942061361209, P111=8.482505866599628e-05, P112=0.00014373349439085936, P113=0.00044921091503522035, P114=0.00025039367574238264, P115=0.0001319036848949202, P116=0.00010279171829501346, P117=8.500959909340264e-05, P118=0.0001247076335301164, P119=0.00011407975008818977, P120=6.676884265906704e-05, P121=7.727173961448741e-05, P122=5.98112462557071e-05, P123=5.99921210646002e-05, P124=6.279055478094444e-05, P125=5.416062046932532e-05, P126=3.649185640104325e-05, P127=8.092178421453824e-05, P128=7.857554510400813e-05, P129=5.256113782296846e-05, P130=3.759462009746204e-05, P131=3.600419220833244e-05, P132=2.2649944687281488e-05, P133=4.207467679239222e-05, P134=3.1454862234661444e-05, P135=1.9828766617002522e-05, P136=2.814338694341607e-05, P137=3.4009669434815327e-05, P138=6.279550803430902e-05, P139=3.5598291667660734e-05, P140=3.785339229811015e-05, P141=4.222129546950041e-05, P142=4.014734847581095e-05, P143=3.7887410444185534e-05, P144=4.3529718336316946e-05, P145=4.7443370610117166e-05, P146=4.493741216394048e-05, P147=4.323680996578648e-05, P148=5.443911341822519e-05, P149=8.270326484982462e-05, P150=9.168305645994865e-05, P151=6.246563193992439e-05, P152=7.423765670313324e-05, P153=5.329007347535088e-05, P154=3.3783372617422166e-05, P155=5.896657816931988e-05, P156=5.520320822771153e-05, P157=6.243362898384671e-05, P158=7.201891621928471e-05, P159=5.429670083056057e-05, P160=4.426148403808075e-05, P161=3.728982507422433e-05, P162=3.090028567203356e-05, P163=3.1234922870088996e-05, P164=3.8923206461109195e-05, P165=3.220362227769048e-05, P166=5.2574122111924097e-05, P167=3.255448582619274e-05, P168=1.8447043214951692e-05, P169=3.3433978554413515e-05, P170=2.448616012936228e-05, P171=2.8149554296608156e-05, P172=2.8546883943727084e-05, P173=3.1091134746543944e-05, P174=2.6091780696275386e-05, P175=1.8165104206789847e-05, P176=4.861515461820431e-05, P177=6.317016889141048e-05, P178=2.3497050672205095e-05, P179=1.1995631908826056e-06),Row( P0=-710.34056704632, P1=35.20663570023698, P2=-6.3595114260505, P3=-8.530169801386949, P4=-19.21758152557356, P5=-11.08905875323287, P6=-10.375693021610154, P7=-3.6279894027326156, P8=-7.663711098771, P9=-9.90644549865409, P10=-2.389194971243024, P11=-6.582031275443168, P12=-1.3831585428534765, P13=1.316210146528106, P14=2.3952712716923714, P15=2.561012160716137, P16=4.799471659696732, P17=3.9840591994394634, P18=2.6562985895550626, P19=2.0297718595474503, P20=0.29288767105066504, P21=1.7492679695126, P22=-2.730516623847774, P23=0.3888347166921427, P24=-5.2372303437823815, P25=-3.958657511310048, P26=-3.687988069022668, P27=-1.8554549575434842, P28=-0.7321921646326229, P29=1.8965471591140728, P30=-1.3532854600520512, P31=-0.34579997096591913, P32=-0.6563531510963652, P33=-0.2902714384045197, P34=-4.13086895867466, P35=-2.4241585178428036, P36=-2.267947347492141, P37=0.764264111872193, P38=-1.5716902162179343, P39=-0.5577140012999586, P40=0.7001893572877192, P41=0.7572621320563584, P42=0.7686684110448896, P43=0.6627454970176943, P44=0.6261275314738759, P45=0.6265236970049122, P46=0.6200615743796258, P47=0.6306062830318979, P48=0.6847063371373632, P49=0.7193918138111969, P50=0.6761039305638739, P51=0.6479014471237157, P52=5.259393753526456e-07, P53=1.7925111056948139e-06, P54=4.334279235886895e-06, P55=4.022212754997187e-06, P56=5.776241679002284e-06, P57=7.647212375241914e-06, P58=4.8245169415049905e-06, P59=4.117753044248239e-06, P60=5.173485191518036e-06, P61=2.9454434157089307e-05, P62=0.0005563560340325518, P63=0.0014159533660395704, P64=0.0019771904549534666, P65=0.005398422583021776, P66=0.009344187799002559, P67=0.013491396970759746, P68=0.029057896697687255, P69=0.07002550761528215, P70=0.04133224558145846, P71=0.0070315348290173885, P72=0.01160224050251618, P73=0.013208537304341773, P74=0.004002527693836392, P75=0.0008249394731765471, P76=0.000870075028968223, P77=0.0012887352495714053, P78=0.0011964822765321145, P79=0.0011514839945215175, P80=0.0013701192862637355, P81=0.0015686174473738036, P82=0.0017983881165394894, P83=0.0020228485139430004, P84=0.003820392729866576, P85=0.006800476227736751, P86=0.010382012196091006, P87=0.008719642924075868, P88=0.004556717148696047, P89=0.0015880543327802046, P90=0.0008401421662971093, P91=0.0013628565252673168, P92=0.0021600716343260088, P93=0.0013786157735646886, P94=0.0011428399360831237, P95=0.0010309789725442048, P96=0.0006913129118945033, P97=0.0010172425176115405, P98=0.000785951219550348, P99=0.0006563076615378025, P100=0.0005308683764659378, P101=0.00017576088963921872, P102=0.00027017757097696376, P103=0.0002954218403788103, P104=0.000283016641189182, P105=0.00034948267519337633, P106=0.00027400622357077065, P107=0.00021974038035290846, P108=0.00013865113436556566, P109=9.371473088832112e-05, P110=0.00010148079552546681, P111=5.765205035974972e-05, P112=8.666738578265613e-05, P113=7.507006285713803e-05, P114=0.00013409430245998416, P115=0.00017127535435662431, P116=9.721370833367744e-05, P117=5.356132487865928e-05, P118=2.5823025392583964e-05, P119=2.0650827017907587e-05, P120=3.347397579883586e-05, P121=4.950014826536309e-05, P122=3.310832625829898e-05, P123=4.8713445734389576e-05, P124=8.326681053011256e-05, P125=0.00011913698190187858, P126=6.285494709026844e-05, P127=4.775665308406083e-05, P128=7.249791428860764e-05, P129=4.175735914264379e-05, P130=7.0905541746584e-05, P131=0.0001191255130015438, P132=2.7568886212205512e-05, P133=2.110543254271747e-05, P134=2.4821286405727976e-05, P135=3.351332097129213e-05, P136=2.3713788030717677e-05, P137=2.676596010303609e-05, P138=4.671989245399143e-05, P139=5.4639131468003716e-05, P140=6.900459774241744e-05, P141=6.651423070723787e-05, P142=4.781945322612454e-05, P143=2.1425887409504403e-05, P144=1.0972695258398607e-05, P145=1.248252557819105e-05, P146=3.048804148030208e-05, P147=7.159671543386493e-05, P148=0.00011748747526455621, P149=8.234680402345407e-05, P150=8.565561540206894e-05, P151=9.787606139488834e-05, P152=0.00012421746459396248, P153=0.00013680997619995597, P154=5.462877994333738e-05, P155=5.865202631681391e-05, P156=5.6110560413661355e-05, P157=6.586517620434406e-05, P158=7.887491185832645e-05, P159=0.00013218397919518093, P160=0.00021067502027855318, P161=0.0005993041384867059, P162=0.0006745358131684649, P163=0.0003794498792423002, P164=0.00028839441636584667, P165=0.00019890122688737518, P166=0.0001371238833274566, P167=6.896093918612171e-05, P168=5.33745765776457e-05, P169=8.918934712647157e-05, P170=9.571709531211494e-05, P171=9.03783643543824e-05, P172=0.00012046460369157254, P173=0.00010169973345926356, P174=0.00010479671990612232, P175=0.00011196985388424751, P176=0.00011339244096449439, P177=7.660034077217744e-05, P178=3.142556222956385e-05, P179=3.2592026389642226e-06),Row( P0=-614.5843588473406, P1=24.10192267765378, P2=-17.550290991032053, P3=-10.327919023294886, P4=-21.152524730575774, P5=-11.944422232262555, P6=-10.256539273558007, P7=-3.1916485106233803, P8=-7.544556472583088, P9=-11.306716192125087, P10=-7.097941799021963, P11=-2.61949894343757, P12=1.845819485420947, P13=4.5859077122751435, P14=8.385471037303585, P15=4.564288854363961, P16=0.42489769833189783, P17=1.322420614593669, P18=-1.7953556354698181, P19=-3.991549508714536, P20=-4.941179047585851, P21=-1.534153212215081, P22=-3.8290729748318992, P23=-3.1523711745132155, P24=-6.226935527011369, P25=-3.9715476635259663, P26=-2.444468970345743, P27=-0.015059339300006205, P28=-0.8871547985727619, P29=1.6671183918512182, P30=-0.1902546128781392, P31=2.155148435030965, P32=-2.04717483158835, P33=0.12788647129065814, P34=-1.8337116358359988, P35=0.5371908531187952, P36=-3.1964044353133003, P37=-0.803550140541588, P38=-1.8617534774502462, P39=0.41243298467217987, P40=0.6590826560733322, P41=0.7053658257012675, P42=0.7014936739365287, P43=0.6655451414420849, P44=0.6284887105830199, P45=0.5518552153131742, P46=0.57587364075083, P47=0.5803533613096807, P48=0.5871219754929087, P49=0.5945811160015197, P50=0.5973098701015817, P51=0.6327837087121315, P52=1.5708734438131194e-06, P53=2.879270335531376e-06, P54=4.99921580571304e-06, P55=2.7619274946493176e-05, P56=6.539604524121276e-05, P57=7.300833315711059e-05, P58=5.690205088038967e-05, P59=3.8760108284341856e-05, P60=3.1815852293259635e-05, P61=3.369794241591792e-05, P62=6.914737877157771e-05, P63=0.00016183591870501182, P64=0.00032917785381527705, P65=0.0016578204366223593, P66=0.008042050729189072, P67=0.0164569517702472, P68=0.027731323989143335, P69=0.0671492672441326, P70=0.07689488541773566, P71=0.07886106193251197, P72=0.049386165917745736, P73=0.03639504977657209, P74=0.04036934073939831, P75=0.018842111322708858, P76=0.030383352752073885, P77=0.07712165079630108, P78=0.06201049619734239, P79=0.02794530427193926, P80=0.0042719130774923255, P81=0.0005660258177112881, P82=0.002219679464640134, P83=0.007757457877352418, P84=0.0110160280904278, P85=0.013402402969189837, P86=0.018444329099681137, P87=0.04187000486142195, P88=0.057516035472365196, P89=0.046320272887367966, P90=0.02447332396609764, P91=0.03175448339628237, P92=0.014080623767933198, P93=0.006000890972480394, P94=0.011575560779073724, P95=0.01660045653841606, P96=0.007845878300581173, P97=0.0014275061289818196, P98=0.001940003504718449, P99=0.0012454779188019426, P100=0.0016236342970469102, P101=0.0013397107570440104, P102=0.001314348760996778, P103=0.0011574010738749283, P104=0.0007848292950016958, P105=0.0022052496276491903, P106=0.002028857496166414, P107=0.0009643643577834189, P108=0.0006141754306936099, P109=0.0005870877233803292, P110=0.0006179926730888791, P111=0.005517904740415533, P112=0.003948216594492999, P113=0.00045431000436149655, P114=0.00015677930714663496, P115=0.00017962799429631322, P116=0.0029220568546158418, P117=0.00566467928965052, P118=0.001116933318824239, P119=0.0015910260219433154, P120=0.0007208028937454916, P121=0.0013324835271158642, P122=0.0007838168354566247, P123=0.0004828326318515279, P124=0.0007932496806660114, P125=0.0010975443231411107, P126=0.0005372378445773809, P127=0.0005263559956173031, P128=0.00032230845671694957, P129=0.0002980091481005314, P130=0.0002221025728856136, P131=0.0001615471908857354, P132=0.000925100354152562, P133=0.0006744461638612969, P134=0.00023918338579776102, P135=0.00015233825707690573, P136=0.0004497501796341522, P137=0.0002557782149130482, P138=0.0002521739566171712, P139=0.0002027082955312203, P140=0.00023212840930000823, P141=0.0001883860249650318, P142=0.0002911998074536003, P143=0.0004087634523366273, P144=0.0003989992708073961, P145=0.00022945295340860616, P146=0.00020388459898669408, P147=0.00013787921611417523, P148=0.00017585696972845035, P149=0.0002489073221808249, P150=0.000226404051468841, P151=0.00021332705062119216, P152=0.00023531569202956169, P153=0.00016257691865919044, P154=0.00011690477058781032, P155=0.00011586256907548759, P156=0.0001329224302171471, P157=0.00018799595783378876, P158=0.00016451059755967834, P159=0.00019594805129163732, P160=0.00035725425221930815, P161=0.00037091623711226206, P162=0.00017851159986058193, P163=0.00019173476336924297, P164=0.00015666996753351691, P165=0.00013076386512453198, P166=0.00010361479172461579, P167=9.067018385712016e-05, P168=8.797963825930473e-05, P169=7.816864108129727e-05, P170=7.413310617107322e-05, P171=7.076716655231023e-05, P172=4.714481396146508e-05, P173=4.4984198459094835e-05, P174=5.3939831963326166e-05, P175=6.538152159278652e-05, P176=7.990796063091579e-05, P177=5.8232888764471484e-05, P178=3.222984418962567e-05, P179=3.643748514115065e-06),Row( P0=-805.2260524521555, P1=40.66247290059016, P2=-6.842146356962618, P3=-9.788947279258526, P4=-13.345828195891544, P5=-5.587424436409164, P6=-8.746835534124852, P7=-2.814486461471377, P8=-5.737523029819835, P9=-8.929374246709607, P10=-5.566563200056013, P11=-1.7126864580016188, P12=0.35331578788073575, P13=2.089473373235873, P14=4.594787855506016, P15=4.03183539323931, P16=-1.0299150583288785, P17=-0.927763409300881, P18=-3.14578329129951, P19=0.029684638456151956, P20=5.210184374155953, P21=6.138400840985884, P22=4.800250377552953, P23=3.9136051692223743, P24=-1.7300670891136147, P25=0.8535748088375179, P26=-1.619830521948401, P27=-0.14528583012358637, P28=-2.3451627045374837, P29=0.6403286126935088, P30=-1.3736758902532111, P31=1.0481442916117762, P32=-3.7223922357162276, P33=-5.4914395355604935, P34=-6.489012445231076, P35=-1.7617259370823755, P36=-2.392592429614811, P37=-2.26656563336801, P38=-4.993076863313838, P39=-0.92005477153599, P40=0.493430139537773, P41=0.5625440979240723, P42=0.6055159148358795, P43=0.5844061350452525, P44=0.5595909948278839, P45=0.5657684876320371, P46=0.6275572183107422, P47=0.6132845862551013, P48=0.5362638849741928, P49=0.5202555636059281, P50=0.5087865514147096, P51=0.4819103763239645, P52=1.5665966432506163e-07, P53=8.582116768413621e-07, P54=1.7340126170719508e-06, P55=5.945707768965525e-07, P56=9.72029244897751e-07, P57=8.058356253919491e-07, P58=3.697532003962188e-07, P59=2.2421933790897973e-07, P60=1.1206009535841763e-06, P61=2.450186614762603e-05, P62=0.0008136142251538826, P63=0.002009026783775968, P64=0.0007738940611604791, P65=0.00029770468482315845, P66=0.0004269042087866875, P67=0.0007420407473087313, P68=0.0017616395697806861, P69=0.004852871030116302, P70=0.007386691154132488, P71=0.0046958614745636834, P72=0.0017011184755502356, P73=0.0004464006288783583, P74=0.00038225407199331143, P75=0.0007893720766423014, P76=0.00032850564717298614, P77=0.00014468047945489604, P78=0.0001067946041735078, P79=7.217857412973021e-05, P80=4.94012633834339e-05, P81=5.683423808196123e-05, P82=9.558699479394578e-05, P83=0.0002471757768362381, P84=0.0006199737072932935, P85=0.0011453973100381478, P86=0.00154985883514338, P87=0.0014180216821316285, P88=0.0014341700042686152, P89=0.0012678786613136166, P90=0.0015718836768406166, P91=0.00208744344625506, P92=0.0007096584939591113, P93=0.00010518229523423461, P94=9.269354668688214e-05, P95=5.560594618881999e-05, P96=1.1490841382707284e-05, P97=1.2277223673151504e-05, P98=1.2922753309492108e-05, P99=0.0002187651329622267, P100=0.0018563860139281481, P101=0.0013134920001447443, P102=0.0003487902438863352, P103=0.00022990707430951514, P104=4.306685712685279e-05, P105=4.12486126755005e-06, P106=3.0431422031349425e-06, P107=3.859209555271067e-06, P108=0.00025691230623227503, P109=0.0009661187583500442, P110=0.0003910090380116731, P111=5.297144848948405e-05, P112=1.605373128531716e-05, P113=4.673014305323202e-06, P114=2.5976671990815378e-06, P115=0.00031419947197892033, P116=0.0008343048538770676, P117=0.0006103129657754666, P118=7.987592358254882e-05, P119=2.1637657002754504e-05, P120=4.210715727082101e-06, P121=4.724269487320026e-05, P122=7.227672112100428e-05, P123=3.9404106817037465e-05, P124=0.00011468928443565802, P125=3.56965860343219e-05, P126=5.087561287232307e-06, P127=8.777580125575144e-06, P128=9.97710391002219e-06, P129=4.0393216026669764e-06, P130=2.8410215578771705e-06, P131=2.9752989894785636e-06, P132=2.2661920926538532e-06, P133=2.092608216961998e-06, P134=1.3857551245393239e-06, P135=1.1611991950657193e-06, P136=1.5202101274179136e-06, P137=3.6388837085896e-06, P138=1.7571837203060562e-06, P139=2.597770275657734e-06, P140=2.720797556707275e-06, P141=5.0762626318978165e-06, P142=6.006216640342914e-06, P143=6.224672728355398e-06, P144=2.681772157598848e-06, P145=2.2396235836119832e-06, P146=3.0229210964132807e-06, P147=2.5979096041562897e-06, P148=7.834331624725515e-06, P149=5.592637816769984e-06, P150=6.273240295917385e-06, P151=5.581186220654159e-06, P152=9.095629447728149e-06, P153=1.0062343968956196e-05, P154=5.9403568309484205e-06, P155=6.3740647641494956e-06, P156=9.672883214008504e-06, P157=9.584745122946005e-06, P158=1.0681313102918038e-05, P159=1.4462567265612001e-05, P160=1.582739960505793e-05, P161=1.5571918737565152e-05, P162=2.763128063044515e-05, P163=4.37836766038873e-05, P164=2.800289978434113e-05, P165=2.5378173393312525e-05, P166=2.7664325109302104e-05, P167=3.357003764738904e-05, P168=2.94358664218437e-05, P169=2.4227190953518233e-05, P170=2.1114325860151726e-05, P171=2.0697268616062365e-05, P172=2.1388820526778652e-05, P173=1.218284580986092e-05, P174=8.099241785752547e-06, P175=7.990189622474586e-06, P176=9.937256975281654e-06, P177=1.0101649568649198e-05, P178=5.609916010773487e-06, P179=5.32704309540452e-07),Row( P0=-681.9302917686774, P1=29.740672036879513, P2=-7.570539033093281, P3=-11.609858499661291, P4=-13.9320039799042, P5=-9.257198247358227, P6=-10.063444879365795, P7=-6.003339985775691, P8=-7.348376762808266, P9=-8.87981387774753, P10=-6.772762787207078, P11=-7.286875048867485, P12=-1.8698692254070484, P13=-0.16697852236892183, P14=0.22519820735166313, P15=3.3893124298496318, P16=2.571741941150903, P17=2.213528746899114, P18=0.26306534883913535, P19=0.18508973249532623, P20=-0.03469540142986782, P21=2.634612502456087, P22=-2.1652119271601133, P23=-1.0692773084069411, P24=-3.4689112948435987, P25=-1.8546206890905463, P26=-2.468383258040852, P27=-1.0972175875449985, P28=-2.941405438124395, P29=-0.24870909946221248, P30=-1.4630926898970151, P31=0.16999762929739912, P32=-1.684195490512998, P33=-0.9530323992234254, P34=-2.64485303539971, P35=-0.10582323630557061, P36=-2.0065515471021413, P37=0.19732721319254423, P38=-1.772683754292729, P39=0.7589296985109268, P40=0.6065272237094237, P41=0.6133191953731898, P42=0.6169985573164638, P43=0.6125534717933815, P44=0.5987068569811114, P45=0.5619670116688861, P46=0.539829978799049, P47=0.5268120305222033, P48=0.5417965291178368, P49=0.5614179485871578, P50=0.5534336952135082, P51=0.5679243310661037, P52=7.59348065175642e-07, P53=2.1366613938591695e-06, P54=2.856819264088529e-06, P55=8.440632728151832e-06, P56=1.8037936474864808e-05, P57=2.329173530178524e-05, P58=1.2988680946980182e-05, P59=7.5637691917693796e-06, P60=1.4823870485291815e-05, P61=0.00013432334969693896, P62=0.000935719470156562, P63=0.0022809147336353996, P64=0.003999805773375259, P65=0.0114928367148399, P66=0.021940310347171837, P67=0.030709514087189787, P68=0.05833740888016862, P69=0.05159162372058599, P70=0.03732626613825561, P71=0.027425001568661032, P72=0.010060667353414592, P73=0.003365735563698448, P74=0.0010950569918600742, P75=0.0012669265649179683, P76=0.0014387034844414435, P77=0.002054146925080755, P78=0.0032143021458271768, P79=0.004285370165601203, P80=0.005539203484257654, P81=0.006524679897092355, P82=0.005395397436892691, P83=0.0033775755525099725, P84=0.005229813306300594, P85=0.00814717371152982, P86=0.008174972229189554, P87=0.01033077079855505, P88=0.011914997341799318, P89=0.014986939304756098, P90=0.015404369553695209, P91=0.006027024076802807, P92=0.0024194467812348208, P93=0.0022604894878076875, P94=0.0018623837523988177, P95=0.0016696019495042699, P96=0.0015618480355736517, P97=0.0013104600465869978, P98=0.0013106656931120115, P99=0.0010789562832592334, P100=0.002105253237043733, P101=0.0019803025259618515, P102=0.0014002119533495308, P103=0.0010270131849621643, P104=0.0005120177973384268, P105=0.001226146046155841, P106=0.0014830942665840635, P107=0.0014884577047609645, P108=0.0011731778504990755, P109=0.0007985712113418919, P110=0.0019649173983574252, P111=0.007658093028603077, P112=0.0016717221865602586, P113=0.00030238534402514096, P114=0.0003433862029543659, P115=0.0006211798398478695, P116=0.00043108137050169973, P117=0.00034333259419924174, P118=0.0004883118074590278, P119=0.00022801844906307915, P120=0.00022647637921324672, P121=0.0005196008038662084, P122=0.0007244559938331479, P123=0.00021585939501299627, P124=0.0003499317923304316, P125=0.00023750795120590178, P126=0.00010367694837439888, P127=0.00017892894121742623, P128=0.00012232902197375283, P129=7.262556976612048e-05, P130=6.826054244519189e-05, P131=8.253825973459424e-05, P132=7.784902715156937e-05, P133=7.670107993652123e-05, P134=0.00012058717307862212, P135=9.516955078123159e-05, P136=0.00014889861451214946, P137=0.0002950391459359639, P138=0.00019695549813973293, P139=0.00013677303397736958, P140=9.214403195091513e-05, P141=7.78385032813229e-05, P142=6.468473520120579e-05, P143=4.207999436564622e-05, P144=4.558960684869919e-05, P145=4.896446386272674e-05, P146=5.123075033909236e-05, P147=7.495499154447566e-05, P148=0.0001228237166849839, P149=0.00015335236654947523, P150=0.00016218369972369952, P151=0.0001685139189846323, P152=0.00012793435228622443, P153=0.0001333490501071539, P154=0.00017101009934306853, P155=0.000189227956824374, P156=0.0001940474207913575, P157=0.00011770653065487215, P158=0.00019955090218647423, P159=0.00018174693594665792, P160=0.0002628407738985189, P161=0.00032781221568554166, P162=0.0003259006856481581, P163=0.000416828563948494, P164=0.00045775080489545003, P165=0.0004168390423692806, P166=0.00048580025021028824, P167=0.000504850553642631, P168=0.0003369793702944953, P169=0.0003535792497380945, P170=0.0005503823251072379, P171=0.0004965296613692172, P172=0.0005068738977674822, P173=0.0005835100926674997, P174=0.0005296992651043129, P175=0.0005583904648696303, P176=0.000622884644915428, P177=0.0005225164930205106, P178=0.0002875172215544729, P179=3.535290396759243e-05),Row( P0=-802.1081573124027, P1=39.25092943464545, P2=-5.271408344266942, P3=-3.8330186031471163, P4=-17.604276516798144, P5=-7.962024122851716, P6=-9.254988510844592, P7=-7.129810303730072, P8=-7.933588226517132, P9=-7.912710164126397, P10=-5.777868630604572, P11=-4.7883429307561105, P12=-1.2810936783542812, P13=-2.791505399976125, P14=0.734481383470415, P15=0.16069863916965935, P16=-1.5725737257951131, P17=3.4344413199469908, P18=2.84466037460618, P19=5.764336685834156, P20=6.920539566446824, P21=5.76881249185876, P22=-0.041198199235938954, P23=-0.8148305781040659, P24=-5.962066069034033, P25=-4.200056613928847, P26=-3.057568083516614, P27=-0.6973473505961836, P28=-2.793473867469723, P29=-0.22472001637869599, P30=-3.2400716238147678, P31=-1.695339017996994, P32=-3.6428213972152657, P33=-1.3769180799315957, P34=-2.7315665282966197, P35=-1.8072795229036134, P36=-2.79492207857825, P37=-0.9882101324775397, P38=-0.04893853497117088, P39=2.878984685219701, P40=0.5485111182724834, P41=0.5397510919716146, P42=0.5579225353809516, P43=0.590130510163724, P44=0.578851495950764, P45=0.5297359061308688, P46=0.5459136355257646, P47=0.6215091661533854, P48=0.6620002764139937, P49=0.6331896299772516, P50=0.5636267958738075, P51=0.527948509043684, P52=2.480882350256084e-07, P53=4.684217117837746e-07, P54=1.3788389921833126e-06, P55=1.9553562798931427e-06, P56=3.1837407082160586e-06, P57=4.2702049613632085e-06, P58=2.3614247177143204e-06, P59=1.7525705290814773e-06, P60=4.229724233436751e-06, P61=2.77242520206399e-05, P62=0.00035016101878794563, P63=0.0021951127791979986, P64=0.004154504551660961, P65=0.003922809925063766, P66=0.0025213308508304825, P67=0.002607688229464359, P68=0.0020454311286569053, P69=0.002140521204842541, P70=0.004335161223904839, P71=0.004156923038812329, P72=0.001152105058396608, P73=0.0002772015222452332, P74=0.00018501026697141754, P75=0.0005134314255125079, P76=0.0024048035346040295, P77=0.003963009431076824, P78=0.0015092936011282506, P79=0.0015099134477884297, P80=0.0007672762712193144, P81=0.00014524945896598421, P82=6.789095329098946e-05, P83=0.0001524299125007613, P84=0.0003263171684592152, P85=0.00023679296305579092, P86=8.042731822336347e-05, P87=0.0001543538477773391, P88=0.0007160008625251475, P89=0.001257525742756124, P90=0.0012541838421909974, P91=0.0006121974243132578, P92=0.0003624293700784706, P93=4.454185354164554e-05, P94=4.274869198662606e-05, P95=0.00014431424275847572, P96=0.00027956403801699645, P97=0.0004651305881633142, P98=0.00024009548753163638, P99=3.592410610575148e-05, P100=4.175787695174596e-05, P101=3.916005391583681e-05, P102=2.0645084279186853e-05, P103=7.714489075198105e-05, P104=0.00015911322077297128, P105=0.0012384065235840158, P106=0.001325212155606556, P107=0.00017214253492061008, P108=2.5438369219527208e-05, P109=1.9041205063542633e-05, P110=7.977723038191834e-05, P111=0.0001600283432156061, P112=0.00015159088349237925, P113=3.8798941109245144e-05, P114=6.490856859766714e-06, P115=1.2836279633069196e-05, P116=5.816552795615051e-05, P117=0.00013868345235058205, P118=0.00012508705260622059, P119=2.864191522909788e-05, P120=8.802751707740748e-06, P121=8.946939672105997e-06, P122=1.2067548488424938e-05, P123=1.2490954115085356e-05, P124=4.528777359746051e-06, P125=6.054298935729325e-06, P126=5.194182470864543e-06, P127=6.612831305333521e-06, P128=5.584678252377988e-06, P129=4.8627910115414336e-06, P130=4.526334100273818e-06, P131=4.58102582236371e-06, P132=4.474178074264823e-06, P133=2.3871537999391334e-06, P134=3.6312955202454025e-06, P135=4.254277830930354e-06, P136=6.120337141000305e-06, P137=3.650793206753114e-06, P138=5.382298981106474e-06, P139=8.17332261140885e-06, P140=9.338284817782902e-06, P141=8.218348660564868e-06, P142=9.61822067599688e-06, P143=8.903938387714575e-06, P144=8.72986093974742e-06, P145=7.891959024290252e-06, P146=5.057034623646713e-06, P147=3.6651366577914936e-06, P148=4.1035297299248926e-06, P149=5.051517022666662e-06, P150=4.472789771909462e-06, P151=6.885887698697886e-06, P152=8.559813626181122e-06, P153=4.735239906259024e-06, P154=4.387443195055478e-06, P155=8.285085480969567e-06, P156=1.1859331737521096e-05, P157=1.797086336804023e-05, P158=1.3536152484610827e-05, P159=1.318186413349513e-05, P160=8.27376306169327e-06, P161=5.78491778226804e-06, P162=4.440398683989234e-06, P163=3.5794515899105357e-06, P164=3.954101636418087e-06, P165=6.215325797431495e-06, P166=4.298095563136355e-06, P167=3.0673424684799934e-06, P168=3.575347213854032e-06, P169=2.24547772528359e-06, P170=2.4560309256198987e-06, P171=2.9093326456446145e-06, P172=4.397069205156003e-06, P173=4.653548812363521e-06, P174=3.820358367838769e-06, P175=3.991529267619916e-06, P176=4.75291469141266e-06, P177=4.037867262750921e-06, P178=2.0295664746931933e-06, P179=2.2169354396757138e-07),Row( P0=-539.7827453113771, P1=-1.985507756233315, P2=-38.77764359517194, P3=-14.521500178181777, P4=-27.257442994900636, P5=-14.323468619068622, P6=-5.337707981758599, P7=-1.4781339502216666, P8=14.330666924789499, P9=3.6677452935894066, P10=9.914556861033986, P11=-1.4986123079044216, P12=-13.599212706895816, P13=-2.677962716810727, P14=-3.813436936507128, P15=3.7437976319190565, P16=-2.0347767469048996, P17=1.8747137577229755, P18=-1.4838492484155277, P19=-0.8464282328377424, P20=1.4481637126433438, P21=2.370359183715837, P22=-1.5644037266234914, P23=2.611331988280057, P24=-0.815746523371812, P25=-3.288903780213833, P26=-1.0096185079223416, P27=0.13396633513934295, P28=-0.9361117206935898, P29=0.6968150824719844, P30=-0.6980621515805213, P31=0.45354890420029653, P32=-0.7902657911122294, P33=-0.6885735254497889, P34=-0.5430715748308078, P35=-1.0583642463416743, P36=-0.46250479633290875, P37=1.9265130569769828, P38=-2.1522069184667774, P39=1.8974078054481456, P40=0.6607240524776117, P41=0.6900912260783376, P42=0.6193803353994175, P43=0.4900022270837402, P44=0.4511801904010552, P45=0.46567488249324657, P46=0.4567713802978677, P47=0.45009463714648484, P48=0.5144607699044538, P49=0.562220696426596, P50=0.5937136586844536, P51=0.6195756700902493, P52=1.0800094379449672e-06, P53=2.0261092663738104e-06, P54=3.441885866987399e-06, P55=5.038108314822068e-06, P56=8.871902346123106e-06, P57=1.2164799428419519e-05, P58=1.1292439378103384e-05, P59=1.1560557230393471e-05, P60=1.611030580206891e-05, P61=3.234082162002932e-05, P62=4.731746139385647e-05, P63=6.163136982737587e-05, P64=5.834735424045233e-05, P65=6.0268833962194136e-05, P66=5.695562508704221e-05, P67=5.654474723205065e-05, P68=5.659126192178383e-05, P69=7.14327085782928e-05, P70=0.00013102617652049447, P71=0.000598426972325187, P72=0.0025395567839909435, P73=0.005738405072287658, P74=0.01872942205895724, P75=0.0258900008962712, P76=0.02230536090455256, P77=0.06062537280588199, P78=0.09512401743674895, P79=0.09736224432035452, P80=0.10352597100345991, P81=0.11694841218399969, P82=0.1959200070498964, P83=0.3774150912861097, P84=0.7334462046662245, P85=1.0592069716891575, P86=1.0809320601884087, P87=1.6422298159117938, P88=1.26478142594745, P89=0.21107430201860364, P90=0.025238425840998044, P91=0.004400189565107774, P92=0.0024451024972065366, P93=0.0015337065558362437, P94=0.003594746167926553, P95=0.003763021953835316, P96=0.006140526775490304, P97=0.13080274129026276, P98=0.463232006546918, P99=0.19165805708516442, P100=0.15975516197480935, P101=0.1706447190839215, P102=0.10731538389351258, P103=0.08292634286291635, P104=0.09110256839283853, P105=0.09109095120064246, P106=0.16553630484805137, P107=0.5827697691730861, P108=1.0882172116129272, P109=0.29828921114651463, P110=0.05390332886783811, P111=0.02851550143153604, P112=0.02364265901385295, P113=0.022898814119568157, P114=0.029264408486623317, P115=0.03293026224027926, P116=0.04373284933193732, P117=0.021706903226982387, P118=0.020891830953714267, P119=0.07460467875755065, P120=0.07604584048762546, P121=0.19732123256231732, P122=0.05939388762982748, P123=0.02409661244897153, P124=0.011743310746262645, P125=0.03708981521355883, P126=0.07512650092910424, P127=0.05222598590850136, P128=0.058696457183464126, P129=0.03390374838886433, P130=0.007866740207353593, P131=0.011095425573295091, P132=0.012483803852411655, P133=0.00761620148722649, P134=0.00483212944137992, P135=0.008034519066064063, P136=0.052958872050993186, P137=0.01529978579542283, P138=0.005786671842723704, P139=0.0026069230776413588, P140=0.009264758493410056, P141=0.013228694924468124, P142=0.0053952978808080015, P143=0.0032940684887621655, P144=0.0028505883749830336, P145=0.0029174934636703896, P146=0.004141002697238328, P147=0.018339685541404385, P148=0.01437481045074784, P149=0.015210822985167958, P150=0.026150489299773743, P151=0.019019903246633996, P152=0.0064243573131330224, P153=0.00516813888050067, P154=0.004576683275585869, P155=0.0014643718374232817, P156=0.0006140675960405332, P157=0.0004213526796474024, P158=0.000243388239846879, P159=0.0002510911814987812, P160=0.00040769572171250065, P161=0.0003299986359244879, P162=0.0003109358538263844, P163=0.00034527442300071585, P164=0.00038495873484741173, P165=0.00045636114693413827, P166=0.0003231117800951876, P167=0.000289602241551929, P168=0.00036785653577454155, P169=0.00038799524391878656, P170=0.00034997103207846974, P171=0.00027097906326869434, P172=0.00022673958839286118, P173=0.00036386829075411815, P174=0.00038490327639594327, P175=0.0003837477092656458, P176=0.0003898252456362029, P177=0.00042055816939657676, P178=0.00021163804948782903, P179=2.341722900949751e-05),Row( P0=-619.1197554190126, P1=25.744009044490912, P2=-26.512943422877047, P3=-18.43199133738664, P4=-26.642991675004264, P5=-13.643983781421106, P6=-12.266150192029556, P7=-6.809356118448111, P8=-6.0213097394576165, P9=-17.030750797172587, P10=-5.600686921492517, P11=-5.64285391546345, P12=1.1277343961549962, P13=7.643158037477802, P14=5.938944618224694, P15=11.149754105080582, P16=1.5299659423754284, P17=1.642100241131757, P18=-1.750468469305376, P19=-2.7574245969431805, P20=-0.981660356157148, P21=-0.7165172998275505, P22=-3.990131881833893, P23=-2.1439725046606264, P24=-2.909013951658432, P25=-0.7652791560389418, P26=-2.244808845022461, P27=0.20235591102184627, P28=-0.40448357488180076, P29=1.511187293458053, P30=-1.4182739963597808, P31=0.27754592830197156, P32=-0.7228204603389357, P33=0.21579056575379502, P34=-0.9753013260674027, P35=-1.2756216885126985, P36=-2.0794753344143424, P37=0.8612526449549508, P38=0.16302278600975292, P39=0.49620175712125375, P40=0.5244248071225731, P41=0.5348940731308885, P42=0.5520203033126938, P43=0.5586924960508649, P44=0.5760942731647594, P45=0.522049607240879, P46=0.428663508250989, P47=0.4105158441629206, P48=0.4421839616577643, P49=0.43960865640783514, P50=0.4392014993583715, P51=0.4735452548129329, P52=2.0023433345895507e-07, P53=5.543499506435417e-07, P54=6.762303373259089e-07, P55=2.0518318811926393e-06, P56=4.216771360621769e-06, P57=3.978738746854717e-06, P58=2.9972059611397246e-06, P59=3.4757430511454064e-06, P60=7.663085094887514e-06, P61=1.9963728572164088e-05, P62=0.00012953484932448452, P63=0.00036867028371789275, P64=0.0005293199105144213, P65=0.0035761115496583834, P66=0.017932179523947517, P67=0.03590953188532062, P68=0.0676953583542582, P69=0.13871629711321787, P70=0.15604663773868416, P71=0.10487364167427948, P72=0.12040520479244626, P73=0.0920594632813208, P74=0.017198378591509716, P75=0.0030098177367502304, P76=0.0013221169508273738, P77=0.0013341270846783857, P78=0.0014313100575289437, P79=0.0015439445508464915, P80=0.003265650472958266, P81=0.008599265493086811, P82=0.015773604016102408, P83=0.051594684133077634, P84=0.13166903973520097, P85=0.14911027101485083, P86=0.15524354805181534, P87=0.2844406821087366, P88=0.29085150689730593, P89=0.15395231297785442, P90=0.19639173835291462, P91=0.2632773891567016, P92=0.16822522232556497, P93=0.055497080488643574, P94=0.03928584691665327, P95=0.03865542484766137, P96=0.05076537379460227, P97=0.042174008261986355, P98=0.06651727230779243, P99=0.031301112180745265, P100=0.014739144682722999, P101=0.005950085096614345, P102=0.003289471788283855, P103=0.006685168837141682, P104=0.023533477323030395, P105=0.027168826506593722, P106=0.025711377932829623, P107=0.007871642915733003, P108=0.0022852523021365772, P109=0.0008565976478704354, P110=0.0013888118126156653, P111=0.00840101498258338, P112=0.017261528830607234, P113=0.011255949726706665, P114=0.004486800998345895, P115=0.0065387786320581604, P116=0.011525360739853196, P117=0.0027630326932486745, P118=0.0007273583566254991, P119=0.0026362615858936665, P120=0.0032870694594302665, P121=0.004283888781337032, P122=0.0018153336906812161, P123=0.0015124380159918355, P124=0.0018942953452322153, P125=0.002294051805235034, P126=0.00324683843486104, P127=0.0013736776861428578, P128=0.0012473619237177525, P129=0.0003517016213963435, P130=0.000385074931681578, P131=0.0005074578598539512, P132=0.0006013783703274884, P133=0.001122222464059103, P134=0.0010823030576954337, P135=0.00019964952901502903, P136=0.00020610284620674633, P137=0.00023514941737761867, P138=0.0001559560423803266, P139=0.0004221914784436039, P140=0.0007138159470792575, P141=0.00024927896390887483, P142=0.0006005854083673957, P143=0.00022329504498114073, P144=0.0001046737539273733, P145=9.04517924363538e-05, P146=0.00018462200438919632, P147=0.00022407302930106904, P148=0.00025999513572908775, P149=0.00041114225719186164, P150=0.00036047986973477777, P151=0.0011428509139763416, P152=0.0005554221322378416, P153=0.0004890748880528369, P154=0.00024732478470550147, P155=0.00045783806753778793, P156=0.00022412051621984312, P157=0.0001610330982811955, P158=9.96015621585993e-05, P159=6.74661515957887e-05, P160=8.82135992300244e-05, P161=0.00013654593081147256, P162=0.00013107246584325713, P163=0.00021847819857870736, P164=0.0003595902583105762, P165=0.00043654176893168035, P166=0.00048508771917952484, P167=0.0005524224604458301, P168=0.0004358302740070293, P169=0.00032998636252663235, P170=0.0002815680668857084, P171=0.00035948080246246407, P172=0.0004113414496510178, P173=0.00039659399516227354, P174=0.0005439811532941087, P175=0.0004022001712217554, P176=0.00035626115411668276, P177=0.0003971421364850853, P178=0.0001717126747117515, P179=1.3321143752604184e-05),Row( P0=-797.7317038267631, P1=31.657044152233635, P2=-5.415676391831566, P3=-5.4252983129308285, P4=-17.227282850543077, P5=-9.874977024385553, P6=-8.423375560534708, P7=-7.648855804157072, P8=-7.147059119622614, P9=-6.490768996527157, P10=-3.4922153870720534, P11=-1.634810927929974, P12=1.0130983214396547, P13=-0.7573562577145053, P14=3.590600384549903, P15=1.753058234087795, P16=-0.33881718587370535, P17=3.0996815495135848, P18=3.4051865648978428, P19=6.527020993319937, P20=4.4016040230235305, P21=1.9646361860694572, P22=-4.6365916551534205, P23=-1.0599769583768088, P24=-2.8344894691716145, P25=0.0878897873759923, P26=-1.1170774666674579, P27=-3.1047221466299995, P28=-4.558614932353471, P29=-1.4553011994272034, P30=-4.251773325750107, P31=-1.935566160360234, P32=-3.170550434025063, P33=-0.4222015659694204, P34=-2.522658103337624, P35=-2.364597708872867, P36=-1.081872706313982, P37=3.666781875618473, P38=2.7691424078199, P39=1.9750035045513261, P40=0.48229773136904674, P41=0.4581953282823152, P42=0.438973783398213, P43=0.44024230318050944, P44=0.5254089183750597, P45=0.4321649228321646, P46=0.40414343908407147, P47=0.43402152284910656, P48=0.5090595527053403, P49=0.5648406499064756, P50=0.5040214414354087, P51=0.4858198064349118, P52=2.7191341245784047e-07, P53=2.7933661326814555e-07, P54=3.63251881268818e-07, P55=8.466865684327117e-07, P56=2.0300707049660744e-06, P57=1.528974814128754e-06, P58=4.852147403855757e-07, P59=3.931258558871918e-07, P60=4.738115955690354e-07, P61=2.171619618816146e-06, P62=1.830535619302774e-05, P63=0.000139944977898466, P64=0.0013854905212743317, P65=0.007688941276511948, P66=0.005109085825987176, P67=0.0012764830395630919, P68=0.0008528513566500039, P69=0.0002885204066247299, P70=0.00021760603611064926, P71=0.0015142458200315884, P72=0.008094749232262069, P73=0.006532263653748122, P74=0.0007197159006782363, P75=6.97344978241174e-05, P76=4.182743314352626e-05, P77=8.748195654558978e-05, P78=0.00031626135630933005, P79=0.0010772069544379018, P80=0.001257257113790597, P81=0.00027970492499178453, P82=0.00014658597662871493, P83=0.00016090495759489193, P84=0.0001718718637119891, P85=0.00010460507961127957, P86=2.0111844006935885e-05, P87=1.6187543967425367e-05, P88=1.4925317015356976e-05, P89=1.869704838230143e-05, P90=8.225642105238937e-05, P91=0.0006241679851947766, P92=0.0011640257932449835, P93=0.00017879381850663997, P94=2.2466862879288954e-05, P95=1.668155632457706e-05, P96=2.4846269857236335e-05, P97=8.217866004735224e-05, P98=3.760237143402766e-05, P99=1.3400101299480552e-05, P100=2.6619498938174498e-05, P101=1.9450718577913618e-05, P102=6.249869220408035e-06, P103=3.063332971831998e-05, P104=0.00014219173857948846, P105=6.384178151161111e-05, P106=5.994311208376941e-05, P107=3.3988835686469346e-05, P108=1.5551613420364048e-05, P109=5.631836220136672e-06, P110=3.4762727477548258e-06, P111=4.944244649020846e-06, P112=1.0268272958635557e-05, P113=1.9083607407802417e-05, P114=5.195118742237385e-06, P115=2.3842643268786677e-06, P116=2.186528160450621e-06, P117=7.062222268828368e-06, P118=1.065223559338453e-05, P119=5.531432617102969e-06, P120=1.0827370299608518e-05, P121=6.9206883795283215e-06, P122=8.697504832101662e-06, P123=7.102606625516537e-06, P124=2.5711333588113846e-06, P125=5.262723392902593e-06, P126=5.8596073477733905e-06, P127=4.805123001015097e-06, P128=3.4500750066508978e-06, P129=3.1668340503021006e-06, P130=4.673086604408774e-06, P131=3.3786487013988696e-06, P132=1.7154592873610638e-06, P133=7.83553357763851e-07, P134=3.3858145486906017e-06, P135=2.264988009765789e-06, P136=1.9763366773958007e-06, P137=2.949737579538631e-06, P138=4.323719008846081e-06, P139=3.2293460526626e-06, P140=3.6259962787054285e-06, P141=6.155027400100916e-06, P142=4.966833602577824e-06, P143=6.5495873544752545e-06, P144=6.202365484837269e-06, P145=4.3625258264089615e-06, P146=1.520204623481992e-06, P147=1.9685763652400722e-06, P148=2.830582291520295e-06, P149=6.3385002640176425e-06, P150=4.444370455031066e-06, P151=2.3067468525433513e-06, P152=2.34957091068317e-06, P153=1.8096656381360872e-06, P154=1.2715020368396972e-06, P155=1.5786495504435196e-06, P156=1.159126106426353e-06, P157=2.1929934420314975e-06, P158=2.8945700542210546e-06, P159=4.8368078168883245e-06, P160=4.854433974432465e-06, P161=5.247958380776592e-06, P162=5.698897685924155e-06, P163=3.424077630450859e-06, P164=3.3125860654286554e-06, P165=2.987262275128525e-06, P166=2.862771461901454e-06, P167=1.6639167764582551e-06, P168=3.2048405607046954e-06, P169=3.604179589600594e-06, P170=2.186414891724607e-06, P171=3.563526081133486e-06, P172=6.7349065515834865e-06, P173=8.995413655011306e-06, P174=4.821886427604103e-06, P175=3.683102397273007e-06, P176=3.0789208720632024e-06, P177=2.3587620346721193e-06, P178=1.4680213871484958e-06, P179=1.6893648721731397e-07),Row( P0=-464.8044513748368, P1=-6.266196711217512, P2=-39.5823759693256, P3=-6.355784920576473, P4=-20.23511924999597, P5=-7.206767404426223, P6=8.352212067729916, P7=4.527625695158207, P8=15.007411624103085, P9=-1.1762283221984793, P10=-2.1613761439504167, P11=-5.030090736726534, P12=-7.599211356673596, P13=8.666713877031505, P14=0.14367778971338907, P15=0.6445377833937358, P16=-6.994561319195031, P17=1.9553647326739947, P18=1.4782195505338909, P19=0.42166244859515845, P20=-0.499173821650676, P21=4.606274947739527, P22=-0.12684458308129104, P23=-1.921742936843197, P24=-1.6596072553435413, P25=0.11788629488549998, P26=0.16384668003692263, P27=-0.4674243279555885, P28=-1.2862132407484057, P29=1.7606064340135479, P30=-0.7242903440972664, P31=1.1448102373296594, P32=1.105151025792722, P33=0.11247484625695355, P34=-1.470215704470296, P35=-0.6719359028094508, P36=-1.0784380719413524, P37=1.1414756267332296, P38=-1.88268153954895, P39=1.4378305413318402, P40=0.6439159875083975, P41=0.6547206746641752, P42=0.6161757250337017, P43=0.6013166146225909, P44=0.5749157022177225, P45=0.5051838594778126, P46=0.43580949537601904, P47=0.42645162953767457, P48=0.4616755487344386, P49=0.5214369735218997, P50=0.5648497517438672, P51=0.5930762453860877, P52=0.00020962804718455876, P53=0.00012493303102737138, P54=8.511993725918079e-05, P55=6.870459535717007e-05, P56=6.072135506326613e-05, P57=7.007051663086401e-05, P58=6.219742752370021e-05, P59=7.06283123531999e-05, P60=9.160126247094338e-05, P61=0.00013931467902581211, P62=0.00021395104616420664, P63=0.00017785742694433187, P64=9.925312599390934e-05, P65=0.00013010961219447775, P66=0.0001612149873007131, P67=0.0001879713016522453, P68=0.00019601331842906763, P69=0.0005629168091793672, P70=0.0007580262902974994, P71=0.0011006376484481064, P72=0.0021052472108392933, P73=0.0021476900823810896, P74=0.0047209045256322415, P75=0.009695967018383874, P76=0.012330278484310626, P77=0.022396641227364466, P78=0.061018675235887526, P79=0.08438257764845226, P80=0.059983380304144907, P81=0.06662330254735269, P82=0.07810640900443062, P83=0.1340327629073037, P84=0.3135697213875312, P85=0.5178809952563916, P86=0.7340903679922176, P87=1.3084219154998098, P88=1.7948882466112956, P89=1.8547595975301974, P90=2.945531301097384, P91=4.028557359337128, P92=4.351813844724972, P93=3.8248276581787457, P94=4.3327402348544375, P95=1.1910329079460158, P96=0.11088623580326015, P97=0.07096333998887738, P98=0.06286686564441527, P99=0.038343767260456504, P100=0.03923656602790561, P101=0.06214543939658753, P102=0.14980459023727855, P103=0.09101213159432793, P104=0.05625176651157848, P105=0.11206428790415184, P106=0.19674048030089672, P107=0.571737401655107, P108=0.6055622776733224, P109=0.3324426089553707, P110=0.8294251242942677, P111=1.6367491980670492, P112=1.5597691446237625, P113=1.1824609832995336, P114=0.9034367396435522, P115=0.2807513006515673, P116=0.0655616566382447, P117=0.04019686921383941, P118=0.03831210611255282, P119=0.05147334787672631, P120=0.06789657469362151, P121=0.10113872564562865, P122=0.3602065184988745, P123=0.7112344465791024, P124=0.5813513969335132, P125=0.3680521963520674, P126=0.14319212873189544, P127=0.14663535157164978, P128=0.15475486169417943, P129=0.02706644612552469, P130=0.030091203237032793, P131=0.08692179261571026, P132=0.228435964459332, P133=0.25150179172372966, P134=0.02687154076005662, P135=0.04477091122772061, P136=0.1024931540568483, P137=0.05172814419829064, P138=0.08158125586475105, P139=0.10564320067723529, P140=0.05486602364619866, P141=0.0157906858804297, P142=0.01565360230791833, P143=0.023591775247030666, P144=0.021341659194598878, P145=0.014977783333953923, P146=0.012183347633529205, P147=0.022729957549798067, P148=0.04467593089287777, P149=0.049652878213669475, P150=0.03178595157368078, P151=0.01147461502329964, P152=0.011897961938850917, P153=0.01576841041720352, P154=0.01076069360134162, P155=0.003136401731621856, P156=0.003194671346852197, P157=0.0016613805820939632, P158=0.0008751651579641738, P159=0.000702499258194708, P160=0.0005605534747013503, P161=0.0004991121818030089, P162=0.0003968181612867165, P163=0.0004629914278798101, P164=0.00036710209793811965, P165=0.00039271403986268456, P166=0.00029286387447535766, P167=0.00042654549497917756, P168=0.0005584953753173739, P169=0.0005983627657845305, P170=0.0010602437193742366, P171=0.0007825491271651434, P172=0.0008003437285404389, P173=0.0011855470114837112, P174=0.0009070947763114301, P175=0.0006769957855083974, P176=0.0007121370085082999, P177=0.000941645572856098, P178=0.0005589452754758606, P179=7.085443722982107e-05),Row( P0=-549.8612010950766, P1=16.924140541894406, P2=-31.378996815906312, P3=-11.882454853193932, P4=-32.22611684240387, P5=-7.4297161222226835, P6=-6.222378577304979, P7=-2.5041571842701047, P8=-1.6772899597186177, P9=-13.706278662096285, P10=-6.080855852888804, P11=-0.263712129976641, P12=-0.9701510808270675, P13=4.722491282676501, P14=-0.27215074681376217, P15=5.8862903825867825, P16=1.9110186288042539, P17=4.55461192496064, P18=3.8880629842636503, P19=0.1454160208630613, P20=-4.04163088474788, P21=-1.9687904297717216, P22=-6.638777152037108, P23=-1.0645574072547133, P24=-2.8051661565721546, P25=-2.61729757137463, P26=-3.657521473022796, P27=-4.7533519454976325, P28=-2.6923561558579996, P29=-0.3660368440085036, P30=1.3799242780279926, P31=0.55123303511339, P32=-0.755221139797185, P33=1.304840853025512, P34=0.5209175808083967, P35=0.36545061584905514, P36=-1.9046401269635753, P37=-0.7072261348949271, P38=-0.15970977174040266, P39=0.7845759546632267, P40=0.5423064601463612, P41=0.5635285107219681, P42=0.5154185197553108, P43=0.5134715348659615, P44=0.538753408495568, P45=0.5440412943140455, P46=0.522467911551331, P47=0.5203441364980935, P48=0.51992180326358, P49=0.46636013397567316, P50=0.5315833526896773, P51=0.5324908925347747, P52=4.548525100104704e-06, P53=4.494948730461914e-06, P54=6.17196460405025e-06, P55=2.4296635815116854e-05, P56=4.747463895684187e-05, P57=4.9130649984959694e-05, P58=2.633738291975267e-05, P59=1.633975011109978e-05, P60=1.816687944195413e-05, P61=3.078823548941647e-05, P62=5.7485450368847836e-05, P63=0.00010265588835198874, P64=0.000325006449797024, P65=0.006997028479145899, P66=0.02645442975717013, P67=0.03493932783165133, P68=0.034118879682743734, P69=0.024754953897922895, P70=0.018277038118645814, P71=0.034782735111498174, P72=0.06562256440082063, P73=0.07982036393486204, P74=0.07488993721400938, P75=0.04318052000795057, P76=0.01029338428384664, P77=0.0017216587158874782, P78=0.001758822474064379, P79=0.002751001552505082, P80=0.008043602959269923, P81=0.014045697876670885, P82=0.021348073163016238, P83=0.03146642180234183, P84=0.09063266748553177, P85=0.1921010209764254, P86=0.061182576705609885, P87=0.011443916550811957, P88=0.00880438436990731, P89=0.036151630623781454, P90=0.05779929633383705, P91=0.1968202254986491, P92=0.4540298789818225, P93=0.1053461653934294, P94=0.06749672111005817, P95=0.04297330727420849, P96=0.16106419863723379, P97=0.22750934615048782, P98=0.1655429763948369, P99=0.017292047137644654, P100=0.0038943284479479637, P101=0.08181788630571707, P102=0.5059812732104786, P103=0.08964875419232624, P104=0.0023659212582377125, P105=0.008557174343867662, P106=0.01749867112378002, P107=0.007589698774737479, P108=0.001426044505697822, P109=0.0014574428304929108, P110=0.011816472919306374, P111=0.05557511329857693, P112=0.007698869514106665, P113=0.002155680731189269, P114=0.002208233470000167, P115=0.004199126515886294, P116=0.0030393291290982103, P117=0.004972337893203539, P118=0.015306795643239515, P119=0.009333163218709173, P120=0.0020101645887138494, P121=0.0010870066234469962, P122=0.002593687919687112, P123=0.01166582292277519, P124=0.0048449343880955205, P125=0.0016622444761317608, P126=0.0014275874653558636, P127=0.003930843163850259, P128=0.0022054192830184105, P129=0.0005897067503645346, P130=0.0009362582455048227, P131=0.00061650577558325, P132=0.0007309836076846211, P133=0.0009945843437109952, P134=0.0011621795630761414, P135=0.0035932850046185008, P136=0.0020706313859009335, P137=0.0027341151664948605, P138=0.0012778782701320542, P139=0.000893161992044947, P140=0.0013153174399716254, P141=0.0011754811768492817, P142=0.0009155396989821413, P143=0.0004882698848469715, P144=0.0008805770332615109, P145=0.0009156109317493305, P146=0.0011054402207859148, P147=0.0021021283914719143, P148=0.001697423079366009, P149=0.0017296694316259308, P150=0.0013954045780886868, P151=0.0011214474119319532, P152=0.0011024772309405515, P153=0.0010793281289979818, P154=0.0007224757958653168, P155=0.0005283441310273011, P156=0.0003154896249737344, P157=0.0002304134370119339, P158=0.000182840711867767, P159=0.00018861517432642607, P160=0.00018353744141458265, P161=0.00017513790595585806, P162=0.00012090485164543776, P163=0.00015279784589804378, P164=0.00018276837553850124, P165=0.0003728562763284144, P166=0.0002779595507049753, P167=0.00022121209265263842, P168=0.00021053447545079076, P169=0.00013954666496249763, P170=0.00015685304743542512, P171=0.00010734267883922751, P172=9.560320862933084e-05, P173=8.047425804851782e-05, P174=7.243286726664076e-05, P175=6.750829513947939e-05, P176=6.359851072451652e-05, P177=5.6471538610465665e-05, P178=2.9357452898635848e-05, P179=3.928982936905945e-06),Row( P0=-672.4490119356337, P1=28.91700603258439, P2=-14.82668269800334, P3=-5.94188976027223, P4=-21.52973448378595, P5=-13.131575495526194, P6=-7.866303659332886, P7=-2.0044416499743885, P8=-7.9549121282395925, P9=-12.094984864225676, P10=0.10953663165267978, P11=-2.459323739134283, P12=2.3192973367446283, P13=7.840908657590901, P14=9.081170061842231, P15=4.341711142078355, P16=3.132745825273761, P17=1.5585417667935764, P18=-0.1516643739070833, P19=0.50637536258603, P20=-1.1683236208333856, P21=1.0601807341120175, P22=-5.227845163949436, P23=-3.510775679191201, P24=-6.246649021519932, P25=-2.411063962079739, P26=-0.9168449613805365, P27=0.24733855569022226, P28=-1.753162786776722, P29=1.1699565668638832, P30=-0.7076789498709704, P31=1.3171296905568484, P32=0.42834743481048143, P33=-1.0961866615462301, P34=-2.0358970134441297, P35=0.4022360671373855, P36=-0.5264960494109746, P37=-0.6541835820916475, P38=-2.8706888971769615, P39=1.3369806840640632, P40=0.6684907507954921, P41=0.6862498014972361, P42=0.7285740730326218, P43=0.6658318768024397, P44=0.6133352950544638, P45=0.548935235830939, P46=0.516937751929165, P47=0.5142883578073716, P48=0.5178230149321719, P49=0.5866089493990155, P50=0.6176652153571589, P51=0.6331802631837613, P52=1.2848085568201793e-06, P53=2.8442009350174996e-06, P54=2.0541273763169052e-06, P55=5.833826563694953e-06, P56=1.3897937942760243e-05, P57=1.2322493180312822e-05, P58=5.990128751442764e-06, P59=5.9558308516426126e-06, P60=5.0379055644021585e-06, P61=7.745558631934303e-06, P62=1.6475108411990415e-05, P63=4.399299996951097e-05, P64=0.00015780620979160247, P65=0.002282101413121094, P66=0.007494537903982804, P67=0.009687939474309166, P68=0.016891754102648312, P69=0.035791077783379976, P70=0.07002736486727712, P71=0.04475228410132725, P72=0.014888363102147808, P73=0.008760827118582323, P74=0.03247552918520081, P75=0.07431201969844481, P76=0.03070193719767674, P77=0.0032910645143026168, P78=0.0003325033357447555, P79=0.00041939095392741195, P80=0.0024552452791747194, P81=0.00362064242882574, P82=0.0026055557330790045, P83=0.0023252339034185213, P84=0.00622099625272998, P85=0.007655334118576462, P86=0.005624706432436489, P87=0.008405684259042218, P88=0.02012601907731339, P89=0.024647388271764898, P90=0.01425775065338225, P91=0.011221866134785876, P92=0.006859270015080867, P93=0.005248787414052576, P94=0.0035253080536237435, P95=0.002247344958871277, P96=0.0028554666789199342, P97=0.0046771934458649505, P98=0.0029241529387972524, P99=0.0015613143899430898, P100=0.003282671453160891, P101=0.004228097005439943, P102=0.0026523630589969983, P103=0.0015475275494196143, P104=0.00046080066087806325, P105=0.0008609397537659555, P106=0.0014632948457022207, P107=0.0003752892119722287, P108=0.00017792103384498914, P109=0.000768385856352115, P110=0.0009167096083849343, P111=0.00031375151790747474, P112=0.0002430801985584539, P113=0.00014492568281103686, P114=0.00017332699901607908, P115=0.00015838421648015237, P116=0.00016942927902638597, P117=0.00032602450827829647, P118=0.00040196133286094854, P119=0.00023114708170054443, P120=4.072160389608789e-05, P121=0.00014232628572455677, P122=0.00043199931008316376, P123=0.00045843473049380116, P124=0.00027401778067575515, P125=0.00015692731349342988, P126=9.044928932523116e-05, P127=0.0003529464138140195, P128=0.00020362500308218796, P129=0.00015966594314682686, P130=0.00015371867940273707, P131=0.00010017396856074486, P132=9.407349055524418e-05, P133=0.00021799089180328784, P134=0.00017100909665262749, P135=9.882754788141142e-05, P136=7.169849666432388e-05, P137=5.5904637749019365e-05, P138=0.00010661976948000916, P139=9.069179867528439e-05, P140=9.679362615237814e-05, P141=9.878183743774068e-05, P142=7.83220763043649e-05, P143=3.8759495993437605e-05, P144=3.447748578523047e-05, P145=2.5113764029851665e-05, P146=2.5655673723606967e-05, P147=4.366914160909006e-05, P148=8.394345406807122e-05, P149=0.0001074893442789303, P150=8.548439368511143e-05, P151=8.67169762153983e-05, P152=9.062564640500745e-05, P153=0.00010721032702398444, P154=9.016754925794398e-05, P155=6.947116850209728e-05, P156=5.962959674134615e-05, P157=0.00010106619065005223, P158=0.00013937100877787378, P159=0.0002538584831909519, P160=0.00026074455872942767, P161=0.00026248863780276927, P162=0.00036719620521902973, P163=0.0004165869867122891, P164=0.00042301190403873384, P165=0.0002671350937407987, P166=0.00022644754295242242, P167=0.00011665012363811417, P168=0.00011416026970203714, P169=0.00021681383641709555, P170=0.00014088673997044186, P171=0.00019291404242845732, P172=0.00011520046496522017, P173=0.00011261379992062914, P174=0.00015323409690661682, P175=0.0001215729088387, P176=0.0001028096244614268, P177=9.720278068557633e-05, P178=5.5774901569857824e-05, P179=3.844000368976016e-06),Row( P0=-490.25629098368967, P1=-7.568780739709104, P2=-49.015238141546355, P3=-7.407065290987045, P4=-26.97902030342538, P5=-13.155522841873395, P6=5.920352797174839, P7=1.8100723405735308, P8=15.600036792667744, P9=2.88259237529592, P10=8.989887177374916, P11=0.0915545176528465, P12=-8.221788485128434, P13=1.3760028608584642, P14=-5.535906808917594, P15=4.0407934102128396, P16=-4.364179450457009, P17=4.5181648489596045, P18=0.15376745044377088, P19=-0.27660318093880626, P20=2.50780446661814, P21=0.516804496955213, P22=-2.157481173314285, P23=-0.7667527667572046, P24=-1.6103454136449658, P25=0.1783184888605723, P26=-1.4750806494341293, P27=-2.0516639675371047, P28=-1.354846833222304, P29=2.2045896261912374, P30=-1.4259504112431047, P31=0.993407494141007, P32=-0.36915585026494147, P33=-0.24321339499464986, P34=-1.9637637128796332, P35=-0.3452125541428311, P36=-0.9976472763446879, P37=0.05305412584723903, P38=-1.0208493632651874, P39=1.494601294237297, P40=0.6408885535222499, P41=0.6385719058451574, P42=0.6320700392142414, P43=0.5882479279446856, P44=0.5588254630548498, P45=0.5363372937127785, P46=0.5370314600087082, P47=0.5390506606963761, P48=0.5739129185871209, P49=0.6075318682724691, P50=0.6439239801238171, P51=0.6207773979262864, P52=0.0003553059209121007, P53=0.00045197497477466074, P54=0.00047658603708900286, P55=0.0001575703850764618, P56=6.472431697021448e-05, P57=6.250777898959434e-05, P58=2.7624659428134736e-05, P59=1.814327897366746e-05, P60=3.0902551686413095e-05, P61=5.433781545958198e-05, P62=6.801154736793618e-05, P63=6.270013989544791e-05, P64=3.389511308864858e-05, P65=4.777760923487596e-05, P66=7.70373196320268e-05, P67=9.62750396433547e-05, P68=0.0002773193361292913, P69=0.0006982524562460663, P70=0.0009314167298932039, P71=0.0014438567662824655, P72=0.0034282518473199744, P73=0.004676539965615171, P74=0.013270972569347908, P75=0.025005543963117105, P76=0.031484592416903344, P77=0.08918986086781537, P78=0.1466416996969674, P79=0.18485121939727486, P80=0.17605433613039612, P81=0.13939606068463228, P82=0.12118133434365397, P83=0.2282191730880601, P84=0.5655311182731093, P85=0.5995516267506882, P86=0.4211802431703344, P87=0.45118859575806564, P88=0.5408707100439234, P89=0.4996250797744365, P90=0.42584478216546284, P91=0.7835580958552268, P92=0.5192998440078598, P93=0.41091914011225295, P94=1.197560797771249, P95=1.9181542602192003, P96=0.6808679540339809, P97=0.07688119408684653, P98=0.1127279161399238, P99=0.22298080293413966, P100=0.25745045442137704, P101=0.2456827989591448, P102=0.4252209701894604, P103=0.6815241089851083, P104=0.3540246476100395, P105=0.24930162899491237, P106=0.39886850493414133, P107=0.6046302982945675, P108=0.6384132305693317, P109=1.1012193985306105, P110=1.3350688416851646, P111=0.3042661806280758, P112=0.3340074095682986, P113=0.46359489742356746, P114=0.4778594952965285, P115=0.4270738767421964, P116=0.32937127794457366, P117=0.06302914359271364, P118=0.11007084751103981, P119=0.16579721265694375, P120=0.24925042976822953, P121=0.13586975966681464, P122=0.2118857334441339, P123=0.7384130043578905, P124=0.49074355220815247, P125=0.12745661527756283, P126=0.09557831395546863, P127=0.1301373664314585, P128=0.11018880583815265, P129=0.09045885116205209, P130=0.11673865819193555, P131=0.09733794542949446, P132=0.11535479395890083, P133=0.09658260890565205, P134=0.14720927250689478, P135=0.08274738668476668, P136=0.09471761400440812, P137=0.041308989027688855, P138=0.03783021948893156, P139=0.05818062380010321, P140=0.11197448538948944, P141=0.07502530336993843, P142=0.03644583152750349, P143=0.03574884105386246, P144=0.06432028010468167, P145=0.03534037977538878, P146=0.01498216668619535, P147=0.015793119993163136, P148=0.0733359267668871, P149=0.0821944933198161, P150=0.048928420709483535, P151=0.045912135803883126, P152=0.03430043956984261, P153=0.01798250555897602, P154=0.010723144086708292, P155=0.00740403470525722, P156=0.002714582925871362, P157=0.00148368310695545, P158=0.0005121980415459422, P159=0.00027312780323466977, P160=0.00020856917266054114, P161=0.00011982107305312464, P162=0.0001584979372910741, P163=0.00022392714169416436, P164=0.000166780912428372, P165=0.00025922506043171375, P166=0.0002261024620453253, P167=0.0001904661868977581, P168=0.00026965140543346884, P169=0.00039712974953703744, P170=0.00044230189264410146, P171=0.00048184139784417703, P172=0.0005452282972476251, P173=0.0005302044648708971, P174=0.0005058089003987022, P175=0.00036666952365135114, P176=0.00044934264040302627, P177=0.0004796226103102079, P178=0.0003229181695102131, P179=2.226505301342113e-05),Row( P0=-505.0614418497259, P1=3.500405543503603, P2=-41.25945607306061, P3=-6.619222134870726, P4=-25.976084389033275, P5=-11.03705739724862, P6=-2.630887312122079, P7=-4.2716863245729195, P8=10.728084264459051, P9=4.893187363624205, P10=16.0762382742502, P11=7.81361938446913, P12=0.21907553974846514, P13=0.7876538450152377, P14=-9.735572585248901, P15=3.4201566648900275, P16=-1.6155404266405184, P17=4.5004311613477554, P18=0.9186058701006418, P19=0.35218092529634193, P20=1.4703922919435244, P21=2.7450886244663995, P22=-0.9411013699510612, P23=1.5420553346961865, P24=-1.8660297295473973, P25=-1.1617304928351915, P26=-0.8194077414562804, P27=-0.6560044086119345, P28=-0.6644048008760775, P29=0.9261833868804342, P30=0.3391550190876836, P31=2.0950620211678688, P32=-0.3173243210130637, P33=0.469403227177732, P34=-1.0737052543453125, P35=-0.18099849066884494, P36=-2.773053076015692, P37=1.2593380201134836, P38=-0.03008585992672651, P39=0.5824016727540327, P40=0.5533711410800665, P41=0.5301269867760439, P42=0.518935498543425, P43=0.4840324553647031, P44=0.502494511169353, P45=0.48458293104780775, P46=0.4980113878449283, P47=0.5335462750909615, P48=0.5857382447122561, P49=0.595472797513291, P50=0.52205363724005, P51=0.5497357828172312, P52=0.002718969307753494, P53=0.004074994708846356, P54=0.0022538110408641156, P55=0.0002920549404565474, P56=0.0001558719092177744, P57=0.00014182358048390212, P58=6.535654338299877e-05, P59=3.0437991896005825e-05, P60=2.954680270298561e-05, P61=3.388869484726722e-05, P62=5.319431443381674e-05, P63=5.808406464640374e-05, P64=3.22987276466611e-05, P65=5.4809762819362333e-05, P66=0.0001869951530745416, P67=0.0003899350539851556, P68=0.000876600952406498, P69=0.0019241874111512165, P70=0.0024605245977774343, P71=0.002172340004577497, P72=0.006087314299594154, P73=0.008428207015138087, P74=0.015668132649062746, P75=0.03499691103305888, P76=0.07783440602685082, P77=0.1364586270158825, P78=0.17003397506445547, P79=0.17290030308864984, P80=0.15576376634099628, P81=0.1564501147644001, P82=0.18826255789600682, P83=0.410982733311246, P84=0.5947430544491035, P85=0.5401255336261028, P86=1.1360241631804122, P87=1.121218237136156, P88=0.1967618623047521, P89=0.05163946496410296, P90=0.023600984727137866, P91=0.04107414005779012, P92=0.1506880629295723, P93=0.0653697419901563, P94=0.06181085197077293, P95=0.07414135225389377, P96=0.09903758816781584, P97=0.22222879000378493, P98=0.3236422097648685, P99=0.9515408884182143, P100=1.427135381809073, P101=0.3945368729508995, P102=0.22417378417620118, P103=0.2627030529994999, P104=0.3201235817702003, P105=0.3306217424834563, P106=0.1679582327853829, P107=0.03687292957514551, P108=0.06312937151927296, P109=0.12918773522042498, P110=0.10383049845739067, P111=0.07619881061869684, P112=0.16491118721636644, P113=0.19283952620825925, P114=0.13068711675065467, P115=0.14610783447236858, P116=0.09705343784565837, P117=0.11808588620427471, P118=0.20533104922545123, P119=0.14205731597782825, P120=0.19764988769714226, P121=0.16429842385863563, P122=0.04308452037146544, P123=0.014080083651229014, P124=0.10654534178102087, P125=0.13339185293126324, P126=0.03459877172734928, P127=0.040339302110125125, P128=0.027638333382847997, P129=0.07357480648017826, P130=0.04296710483196867, P131=0.031421986346378994, P132=0.010518723027201003, P133=0.04976428703931203, P134=0.02765678848331088, P135=0.04194425449731712, P136=0.05797160136799158, P137=0.02370956372413318, P138=0.03854442835612041, P139=0.026788851792418064, P140=0.016908060681810968, P141=0.019948225048108625, P142=0.024123670970257558, P143=0.010870526611656035, P144=0.0029362303971147284, P145=0.007247061680087073, P146=0.0171615897035592, P147=0.008114837046379043, P148=0.03184226659262044, P149=0.06344760960863918, P150=0.027402323543615718, P151=0.013796423257789365, P152=0.014380623894843179, P153=0.006430524748962235, P154=0.007824621250289725, P155=0.003932263573429441, P156=0.0028372406943499347, P157=0.001514346204573074, P158=0.0005527082473341757, P159=0.0007094236758925214, P160=0.0004385875472579688, P161=0.00029751928753218073, P162=0.0003981722162154245, P163=0.0006628530165042458, P164=0.0007644385074298337, P165=0.0006526639874363117, P166=0.0006999148853194067, P167=0.0005981006121466968, P168=0.0004924585218360347, P169=0.0003871747867750391, P170=0.00032434129444612044, P171=0.0002104737667198384, P172=0.000381273348986781, P173=0.00036103624193513264, P174=0.00029409018998156493, P175=0.0002760585558394016, P176=0.00023915212338337588, P177=0.00027069884329764546, P178=0.00014523859613718184, P179=1.3368492236276453e-05),Row( P0=-740.490674907971, P1=24.2304911374438, P2=-4.904861240098787, P3=-8.928193944311907, P4=-19.25860070769523, P5=-9.395949198644775, P6=-12.183413632703402, P7=-7.572650420914523, P8=-9.352608434778668, P9=-9.637717657364101, P10=-5.043371833976513, P11=-3.45132196710348, P12=0.5842386730701602, P13=3.224670251819219, P14=7.829000733103589, P15=6.021006419194377, P16=2.3398218815322385, P17=-0.6458243241975518, P18=-1.940815058156714, P19=-0.9322840848247685, P20=2.242987606203721, P21=0.9797351460448573, P22=-0.6522548513401663, P23=-2.028198842641888, P24=-5.9493256035195605, P25=-3.9104457684918468, P26=-1.5175259146870368, P27=-0.1982550101360406, P28=-1.5469218892742571, P29=1.541640209374846, P30=-3.2569468141548814, P31=0.18797391381317502, P32=-2.3082714025542996, P33=-1.637182414653711, P34=-2.993335476216531, P35=-1.2328967832813365, P36=-2.8859331874467595, P37=0.40510062940732905, P38=-0.9014739507347519, P39=1.9479799912874647, P40=0.46518501877873414, P41=0.5056016909706597, P42=0.5525513106621919, P43=0.5199474843020473, P44=0.49815864266423376, P45=0.44518621347780324, P46=0.4122214053701839, P47=0.4562009070036441, P48=0.47459610647652734, P49=0.4508568549837449, P50=0.43661582643563157, P51=0.4503841049548844, P52=1.3806979729652656e-07, P53=3.032257924174842e-07, P54=9.61272841513543e-07, P55=3.927824911823039e-06, P56=6.764900337316249e-06, P57=5.9629689047951575e-06, P58=2.5876782362487482e-06, P59=1.6930694510957172e-06, P60=1.3269272297959045e-06, P61=2.4264505618952517e-06, P62=8.761372188022105e-05, P63=0.0008447779721995333, P64=0.0015351743564742566, P65=0.002235478481124017, P66=0.002847338938888225, P67=0.005161964305857982, P68=0.01619134284582993, P69=0.048032318512013776, P70=0.03157464949564183, P71=0.0103917510312207, P72=0.008327779149867738, P73=0.003253990871950065, P74=0.0004376737603174445, P75=0.000206402503560667, P76=0.0002328042152960366, P77=0.0003442715799580685, P78=0.00045603818787805445, P79=0.0004182857078951715, P80=0.0001833566305656931, P81=9.014974526493003e-05, P82=7.456648171769554e-05, P83=0.00023987722725151269, P84=0.0008853053672864676, P85=0.0015967241492374075, P86=0.0026423683056621874, P87=0.004669449636563277, P88=0.004060342605677576, P89=0.0017512172762033788, P90=0.0019417848939874702, P91=0.0043823229581505556, P92=0.0009992025294786433, P93=4.14152506327044e-05, P94=1.2279247957740832e-05, P95=7.908993684620942e-05, P96=0.00023536763664690736, P97=0.0005579999086599874, P98=0.0005392780523245781, P99=0.00016422274905363222, P100=0.0006642120902252361, P101=0.002008739003134108, P102=0.0022614957250083802, P103=0.003962827874166471, P104=0.0017541589649847375, P105=0.00015839507120271663, P106=0.00021958204914989765, P107=0.00010025395206466829, P108=2.826419122860131e-05, P109=0.00011126060635866724, P110=0.0004624061500288005, P111=0.0005778572981528069, P112=0.0006256291205094979, P113=0.00021525305291807757, P114=4.7668840295755195e-06, P115=6.0130944989477285e-06, P116=5.894709857962382e-05, P117=0.0003537326629002976, P118=0.0005309648059999456, P119=0.00036924380405454224, P120=4.1429791409345846e-05, P121=1.7180116276495083e-05, P122=5.300444096180798e-05, P123=2.714708213615211e-05, P124=2.0552740321152413e-05, P125=2.6146575136146268e-05, P126=9.302432543951862e-06, P127=2.336256450828692e-05, P128=2.2186447892654237e-05, P129=1.266402533342794e-05, P130=3.409221651377186e-05, P131=3.222502029834181e-05, P132=1.1033461753225893e-05, P133=9.991368519017232e-06, P134=8.26916575529376e-06, P135=3.0290710767785775e-06, P136=5.809060526929693e-06, P137=1.1516504190743034e-05, P138=6.796620936269461e-06, P139=1.0715619521882028e-05, P140=1.9208549672267517e-05, P141=2.095153059593209e-05, P142=2.3696219129600297e-05, P143=2.3452552031217164e-05, P144=8.949448704945932e-06, P145=8.764101333896358e-06, P146=1.2991095924748803e-05, P147=2.1190997117061975e-05, P148=5.2994827434361494e-05, P149=3.418431122975876e-05, P150=2.78403957200162e-05, P151=2.7985375715594088e-05, P152=2.811510392512659e-05, P153=2.2573361287518534e-05, P154=3.1612569196536565e-05, P155=2.1138908582079493e-05, P156=2.8211396754854402e-05, P157=6.281419682979914e-05, P158=7.867234723961131e-05, P159=0.00011574017700588021, P160=9.110341530098628e-05, P161=9.563666872965128e-05, P162=0.00010775200321036411, P163=0.00012046653722630897, P164=0.00011059508702077097, P165=8.98422659070134e-05, P166=6.46801603823287e-05, P167=7.461527020812465e-05, P168=6.733836843878186e-05, P169=5.9374537186753465e-05, P170=6.438493216393575e-05, P171=5.123535158302794e-05, P172=3.702129879237989e-05, P173=4.226530092564474e-05, P174=4.0943740309736816e-05, P175=2.945947729746776e-05, P176=3.814991147763687e-05, P177=3.4364477994695296e-05, P178=1.3119428331549214e-05, P179=1.7292224228090026e-06),Row( P0=-625.5490607580801, P1=20.791156355862984, P2=-22.01326416315955, P3=-10.050362828847659, P4=-24.361343188532015, P5=-15.267077360075795, P6=-8.587823906251074, P7=-7.199714668073904, P8=-10.318347561073223, P9=-12.218504709794008, P10=1.2186200187885423, P11=0.25462053902859144, P12=6.622730259834171, P13=7.981562780502894, P14=2.648134489815002, P15=-1.0058854668948207, P16=-3.08266519600284, P17=0.447977630750193, P18=0.8438637207747897, P19=2.7479907532812784, P20=2.002989180415308, P21=2.911238982104298, P22=-2.142993699946601, P23=0.42998602756749316, P24=-3.7436469576354945, P25=-2.212411152309771, P26=-1.3190742437684286, P27=-0.49457384842900476, P28=-1.051182207679231, P29=1.8052084767901826, P30=-2.787168841328043, P31=0.6855785205306002, P32=-2.2168393617481015, P33=-0.1934730704272212, P34=-1.7227726531781637, P35=-1.5524431497685176, P36=-0.7109266371597726, P37=0.8500530810812285, P38=-0.021249193054067576, P39=0.4892527778753865, P40=0.5283269194799946, P41=0.5935515713849124, P42=0.6205840919155778, P43=0.6000123867829623, P44=0.5952781484079465, P45=0.5802420892817118, P46=0.5550504811018283, P47=0.5473877412793352, P48=0.5429775227070195, P49=0.5817129151740058, P50=0.5477578641926957, P51=0.5107143065579804, P52=6.294783212618275e-07, P53=1.9360131719607782e-06, P54=3.935181122952188e-06, P55=5.65366982022865e-06, P56=1.0323851949736462e-05, P57=9.110239034884108e-06, P58=3.91995297833028e-06, P59=4.459770215618816e-06, P60=7.768853945876838e-06, P61=1.070749328450462e-05, P62=0.00014152643792971167, P63=0.0011003562383180888, P64=0.002061604437609376, P65=0.005571802167702426, P66=0.008286506437982875, P67=0.007329135274761908, P68=0.012547704188689668, P69=0.03959172557811537, P70=0.04726913679676292, P71=0.08947278088318909, P72=0.1534978206998771, P73=0.09466821911924242, P74=0.059721624259756204, P75=0.05414878146280884, P76=0.06615892664072845, P77=0.08754166321134625, P78=0.06754130488861003, P79=0.03747197506353084, P80=0.005455977760980247, P81=0.0010281548066998487, P82=0.0007043219738642241, P83=0.0010361059101929053, P84=0.002056270222034974, P85=0.003265720864570803, P86=0.007723434571597365, P87=0.03209212133090256, P88=0.038049603916485826, P89=0.018058619991857358, P90=0.01976579259054965, P91=0.04151718676401624, P92=0.08218035510883354, P93=0.1739126606284621, P94=0.1122644379105245, P95=0.08688788521165323, P96=0.03748872852996022, P97=0.039632058203793626, P98=0.05660308945197902, P99=0.14166168459506068, P100=0.18080099506711042, P101=0.01748152373876639, P102=0.0031972036672622156, P103=0.007721422256567228, P104=0.01637435083899479, P105=0.007706702666682748, P106=0.012728781931064075, P107=0.024924941800525107, P108=0.007714558181534301, P109=0.002664218141230688, P110=0.0033812302399251807, P111=0.003191691244628705, P112=0.002917933983020453, P113=0.004611129606945724, P114=0.004105968372544105, P115=0.004251344693981902, P116=0.002430991032595292, P117=0.001161806248452804, P118=0.0011950100174975946, P119=0.0015171875764024387, P120=0.0021442382431403075, P121=0.00835285615631715, P122=0.0027104578869272225, P123=0.0009112462609199066, P124=0.0006996959894458601, P125=0.0032788738984130573, P126=0.00432495117183817, P127=0.0009809472401111953, P128=0.0006010260661214617, P129=0.0007284530142349536, P130=0.0009301857122446833, P131=0.0007250251410480439, P132=0.0005308648734142483, P133=0.001647398184152696, P134=0.0018121941324872987, P135=0.00025488962100124165, P136=0.00042998637935209844, P137=0.000504805910171534, P138=0.0002695061571868019, P139=0.000191287891478663, P140=0.0003482514230852862, P141=0.0002485656976806583, P142=0.00021420333496665326, P143=0.00017247887156945455, P144=6.557293087153447e-05, P145=5.395999142948622e-05, P146=5.448190554895876e-05, P147=6.880263343282919e-05, P148=0.00010851678777761883, P149=0.00026380948406607986, P150=0.00018033601984069068, P151=0.0003322804703631298, P152=0.00018800918640691206, P153=0.00013849799709113764, P154=6.995521433188097e-05, P155=0.00010355582644649135, P156=0.00010899950955714183, P157=0.00011964607486873845, P158=0.00023398097507824835, P159=0.00014839459123126728, P160=0.0002006264992292671, P161=0.0002886401616480123, P162=0.00024603979418939484, P163=0.0002528756635420676, P164=0.00028222148397340967, P165=0.00030815134893051317, P166=0.0002526149751028385, P167=0.00021944212762528068, P168=0.0002691847526263877, P169=0.00023901723085411654, P170=0.00018495705301616375, P171=7.35856062585255e-05, P172=8.928274876837215e-05, P173=0.00011102644584230534, P174=0.00010041024836778762, P175=7.387962422766605e-05, P176=8.821769801304371e-05, P177=6.397669295735714e-05, P178=3.046849861164603e-05, P179=3.1306480297833558e-06),Row( P0=-632.4977556365567, P1=25.895812687066925, P2=-23.071998385942155, P3=-12.305507775344187, P4=-26.52367296947685, P5=-9.554213857233183, P6=-11.1625932279861, P7=0.17479838045058985, P8=-2.648973398627331, P9=-14.477015409021208, P10=-5.29520597427614, P11=-7.943639897475051, P12=-0.11000325652442368, P13=-1.9593289995214163, P14=2.1206667339212744, P15=5.764488219999765, P16=-0.23434799531528797, P17=4.480633314767203, P18=3.2085319508610333, P19=1.6472666839155712, P20=-3.0410610294049185, P21=-0.5385359627885258, P22=-6.321159742893569, P23=-2.0632714585133405, P24=-4.626223533834649, P25=-2.30081916310672, P26=-4.2022594357660035, P27=-1.4660479045148438, P28=-3.3118487724715755, P29=1.2419852640799642, P30=-0.11646918179135587, P31=2.5116201098675313, P32=-1.0985786128317625, P33=0.8548784196600485, P34=-0.6861713286863246, P35=-0.19833134909571312, P36=-1.5480831450272095, P37=-0.3990056278231109, P38=-0.637285796256242, P39=0.8496184024968203, P40=0.6072542223124203, P41=0.5806347005704521, P42=0.5569662549019669, P43=0.5672112427917687, P44=0.5966553681921066, P45=0.5532699841507718, P46=0.5457344747929305, P47=0.5619677645895084, P48=0.5884160238586307, P49=0.5728731243103846, P50=0.5977622917280807, P51=0.6093977849504396, P52=8.360839097743909e-07, P53=2.0201662587311105e-06, P54=4.280919536059422e-06, P55=1.4149852579891377e-05, P56=2.1303842288937852e-05, P57=1.933494991299132e-05, P58=1.1320852445695026e-05, P59=1.1156388888374873e-05, P60=1.7173162259279878e-05, P61=3.688295910645282e-05, P62=0.00013829056565890725, P63=0.000408271665712848, P64=0.0012885930159884601, P65=0.004296703403502199, P66=0.01344944090084579, P67=0.020634214379144068, P68=0.024476183923344417, P69=0.01580050849687151, P70=0.007889600934489534, P71=0.012132597490819337, P72=0.01227872485185753, P73=0.004554813242542773, P74=0.000690548751193342, P75=0.00040713650379729793, P76=0.0004075404480163142, P77=0.0012214269856974987, P78=0.003884612798298354, P79=0.005636530450172275, P80=0.0060189215613642546, P81=0.012521316754530918, P82=0.01694962213456283, P83=0.023068577892792038, P84=0.03072033369094925, P85=0.02864629698382988, P86=0.01778556449672886, P87=0.010642621093175673, P88=0.005541023865475938, P89=0.006543795338833783, P90=0.027747868380559905, P91=0.08812257472851633, P92=0.05420296557194565, P93=0.06167848489259495, P94=0.05523773792379608, P95=0.040788917340614336, P96=0.02413007596809741, P97=0.01095552830118623, P98=0.0028642602133538983, P99=0.0022800980114360894, P100=0.004309224148235379, P101=0.0019443209604490046, P102=0.005389433287026866, P103=0.0037646717325598685, P104=0.0014350273993664726, P105=0.0007451305541867108, P106=0.0012055396828870423, P107=0.001926293807097253, P108=0.0012033702574326891, P109=0.0008673555563269434, P110=0.0009124607768488306, P111=0.0011879205236670835, P112=0.0016457832280572537, P113=0.0011187975033133827, P114=0.00078385570459477, P115=0.0015983789494743853, P116=0.0018843195362659058, P117=0.004257631799204832, P118=0.0037248867116852153, P119=0.003391786461739334, P120=0.004889180962998954, P121=0.005324549198031491, P122=0.0008860717143551946, P123=0.001365146882544314, P124=0.0032195547738851885, P125=0.001343847396665957, P126=0.0006615209769459241, P127=0.0024051844859624785, P128=0.00044652285914252105, P129=0.00020085709752127985, P130=0.0005281306387185559, P131=0.00035102480402856803, P132=0.00012342785048206095, P133=0.00018251565124711308, P134=0.00015940166318570445, P135=0.000942034461554074, P136=0.0009749926214023939, P137=0.000356094479260689, P138=0.0003642191337086283, P139=0.00022108291998263583, P140=0.00026846308813440454, P141=0.0002630659947832059, P142=0.0003012589585892717, P143=0.00025517458298619507, P144=0.0003142546463749067, P145=0.0001704920915067232, P146=0.00010506751171617371, P147=0.0002079272379197648, P148=0.0002564844444340761, P149=0.0001732854201930785, P150=0.00031996488959702896, P151=0.0004920934793418567, P152=0.000540918831878071, P153=0.00027083171392450303, P154=0.0002594677892527115, P155=0.0003618561408339669, P156=0.0004459958435282946, P157=0.0002268470688100485, P158=0.00012064999854889251, P159=0.00013146608141912777, P160=0.0001367942645603676, P161=0.00015145446846695856, P162=0.0001780401932968039, P163=0.00021497475691271506, P164=0.00014129454349156475, P165=0.00013847383020175458, P166=0.0001693051337485761, P167=0.00011318659649538458, P168=6.052655644918233e-05, P169=7.330695804044578e-05, P170=5.288393622773962e-05, P171=6.353596243371825e-05, P172=8.639100372041922e-05, P173=0.00010569898808703018, P174=0.0001180636147223422, P175=9.412035106504815e-05, P176=0.00011854867091252842, P177=0.00014651740437085942, P178=6.14362926514528e-05, P179=8.497205681043295e-06),Row( P0=-562.0863536196924, P1=19.750776923228752, P2=-25.60551950772474, P3=-2.72889570735038, P4=-26.20231756586403, P5=-10.947404929373935, P6=-2.256077064082858, P7=-7.4357680534184025, P8=3.5435477000415134, P9=0.012400068291481148, P10=13.190538140871984, P11=11.216971277086685, P12=8.383045496047494, P13=8.79494576274907, P14=-4.291315405715343, P15=1.4790178756478884, P16=-1.2001783431011923, P17=4.985272133605994, P18=2.6705269725292085, P19=1.2371942979702946, P20=-0.23950506763926996, P21=5.051132598961569, P22=-0.25457894225833566, P23=-1.9459361479133175, P24=-6.776644404535365, P25=-3.9440039254659767, P26=0.5754363736844939, P27=0.3378356196397528, P28=-1.1149098011611265, P29=-0.12409125001986451, P30=-2.579462389570794, P31=0.4905857854020829, P32=-0.6963560125470645, P33=-0.1379662732553027, P34=-1.7272970171246118, P35=-0.5683820113763863, P36=-1.433909299748188, P37=0.9357914994296183, P38=-0.9955833989920111, P39=0.8597886139776154, P40=0.5683074171601923, P41=0.6243471276248508, P42=0.679196166685981, P43=0.6776023018886559, P44=0.6844700588225997, P45=0.6327777558679659, P46=0.6325524010650692, P47=0.6445545369824104, P48=0.7164397075411266, P49=0.7440428766950798, P50=0.6687540053597478, P51=0.6028257220906981, P52=0.00566623075674796, P53=0.00505499020384395, P54=0.00226917898134196, P55=0.0005946905714273561, P56=0.00041161046255394135, P57=0.0003235267337129145, P58=0.00010056836336084219, P59=4.467508195650078e-05, P60=1.7550998730732975e-05, P61=7.967244623226976e-06, P62=1.1166020260979932e-05, P63=9.504762942142637e-05, P64=0.001039352708443245, P65=0.0020930755297610603, P66=0.002366052257858135, P67=0.004059326089395602, P68=0.021797086204563314, P69=0.05583664448705657, P70=0.06149511456738971, P71=0.08816915315615387, P72=0.07007251519272585, P73=0.042496344392635034, P74=0.06349811619373395, P75=0.11043418524396231, P76=0.12865526374565, P77=0.2123396670969107, P78=0.33818390625040473, P79=0.4339448246199365, P80=0.3151312347178473, P81=0.22946916841232015, P82=0.1364811949220822, P83=0.016471609241678405, P84=0.0033898899562134098, P85=0.007579205391014307, P86=0.03580613082544255, P87=0.03179966437825961, P88=0.013571344436621443, P89=0.01462415829444649, P90=0.0255375585045834, P91=0.02572311074345557, P92=0.017766557247856495, P93=0.02227620016803933, P94=0.053392098511737165, P95=0.046555936638762604, P96=0.037372465918077545, P97=0.13460727802230643, P98=0.320692248518525, P99=0.22583120461406478, P100=0.12436825180573621, P101=0.04030628218412506, P102=0.03130944463943848, P103=0.022041971261575973, P104=0.002328073813755475, P105=0.006494295206534488, P106=0.0687360716970937, P107=0.16489006516416013, P108=0.060427700775368645, P109=0.04734799769007979, P110=0.053168215449357015, P111=0.08498143749208642, P112=0.052498393740147274, P113=0.01382422279793854, P114=0.016096889048745934, P115=0.03143904140782095, P116=0.020526426324875207, P117=0.005186438404400464, P118=0.011813985855271132, P119=0.013949989147300038, P120=0.021106613465884, P121=0.008555729192519826, P122=0.0054891778683814705, P123=0.005382752285289759, P124=0.018096488072129213, P125=0.01973395500637678, P126=0.0023422801710419002, P127=0.0015701446755897977, P128=0.0037950578473088317, P129=0.004560753276009622, P130=0.0020314198048222614, P131=0.0022046925011237105, P132=0.0031927518727607267, P133=0.004000942550743942, P134=0.008327817884048185, P135=0.010591947878587474, P136=0.00857752576446488, P137=0.004271152463759255, P138=0.004178124118657784, P139=0.0039517945156074125, P140=0.004211163102623231, P141=0.005673079442076864, P142=0.0025245804160669095, P143=0.0014107359815223167, P144=0.0014298068462289472, P145=0.0018050967211759924, P146=0.0021813683485022756, P147=0.002799495019798649, P148=0.00549837207787387, P149=0.0060384809125532905, P150=0.006641100006046834, P151=0.0038256863198005086, P152=0.0021293659009887643, P153=0.0008115455374203131, P154=0.0005268453359855582, P155=0.0004310943451135602, P156=0.00031955256908870223, P157=0.00020467548748873376, P158=0.0003369318364783855, P159=0.0008652139226760693, P160=0.00021818470134523719, P161=0.00012379577772816895, P162=0.00018484743005674008, P163=0.0002949349485282247, P164=0.0003943031274374312, P165=0.00031037633215510036, P166=0.0002790776044407004, P167=0.0001636097339990821, P168=0.00018277413198224045, P169=0.00019151254238849836, P170=0.00014632113250194142, P171=0.00011736251133372005, P172=0.00016874298142237462, P173=0.0001364854252874876, P174=0.00011828381781685176, P175=9.322197018609573e-05, P176=8.279976568079941e-05, P177=8.93268002197721e-05, P178=4.1072323951288635e-05, P179=5.201466350892557e-06),Row( P0=-752.182660547744, P1=30.74817522115097, P2=-7.178378752571702, P3=-7.779100546881567, P4=-20.388914243203057, P5=-10.676069907534682, P6=-11.682018415514504, P7=-8.974894430094494, P8=-6.961952159068225, P9=-10.22280665515577, P10=-4.1709831291852755, P11=-4.6329003455814775, P12=-1.968606949597093, P13=-1.8893516527090979, P14=5.36128438547699, P15=5.6730435284111325, P16=6.631568448253884, P17=5.420180466744208, P18=2.6652797500165177, P19=2.277797103253799, P20=2.7940897640868547, P21=-1.3725800069215326, P22=-2.499996001898215, P23=-3.0537607235750945, P24=-7.723997126796435, P25=-5.444300076339591, P26=-4.813993772414783, P27=-1.379085184064365, P28=-2.871869091300552, P29=-0.4622699456775395, P30=-2.4345679925382764, P31=0.5553735411664866, P32=-0.71697074473422, P33=-0.2007195273266745, P34=-2.942930394800203, P35=-1.0571969564777615, P36=-0.4081585744452161, P37=0.8715348330780246, P38=-2.068183297020973, P39=0.20722422472515958, P40=0.5564272474093108, P41=0.5527288022895813, P42=0.5685915912108578, P43=0.5015388207372775, P44=0.47751764512063954, P45=0.4640678983885419, P46=0.4446273794169342, P47=0.468329522862281, P48=0.5569409199170763, P49=0.6065161414206555, P50=0.5591076825253413, P51=0.5559884743160609, P52=2.2251685907085669e-07, P53=3.0807563590007e-07, P54=6.795253245273391e-07, P55=3.1704836973824472e-06, P56=8.285566372378378e-06, P57=8.27668216308351e-06, P58=3.558074728130438e-06, P59=1.7913473596733468e-06, P60=1.2798620087298149e-06, P61=3.5622493481106994e-06, P62=3.05690318856303e-05, P63=0.00034644545756533444, P64=0.0016595062481710118, P65=0.007848082377226208, P66=0.020868597313077986, P67=0.030644116550672736, P68=0.019474601661138364, P69=0.03174466084133584, P70=0.018328224143005355, P71=0.0061891567078011, P72=0.00298648614578538, P73=0.0010242618016954568, P74=0.0002531711139335266, P75=0.00018344006493427014, P76=0.0002589537011132735, P77=0.000547457499253804, P78=0.0006133529098281608, P79=0.0008972031584773527, P80=0.0016230842030064776, P81=0.001402145746725723, P82=0.0015294499535362396, P83=0.003092557975186745, P84=0.0029408047314334563, P85=0.0025214720332917455, P86=0.0034411638069860835, P87=0.006112624903061914, P88=0.0032095565102901132, P89=0.0013914796453408243, P90=0.000982185416893889, P91=0.0007604795774909486, P92=0.0005978215836574449, P93=0.00025595211988825936, P94=0.00016774622308323743, P95=0.00017540286865359143, P96=9.710103529345942e-05, P97=0.00015268346937523118, P98=0.0005363156535558376, P99=0.012077955346080953, P100=0.006383415592668488, P101=0.00035756537210277086, P102=9.951206949697456e-05, P103=4.982438686746332e-05, P104=4.3350295516408846e-05, P105=2.5026460702909253e-05, P106=1.5334292416104222e-05, P107=0.0003735363824852252, P108=0.006821016591811077, P109=0.001490733591045333, P110=1.6098005575241993e-05, P111=2.715602918424856e-05, P112=9.356784913858891e-06, P113=1.119348966316552e-05, P114=0.00017726335663216966, P115=0.0018609254545256648, P116=0.0002257622688002672, P117=2.578988681958028e-05, P118=2.051691569788068e-05, P119=1.568951118421634e-05, P120=0.0007888803243410798, P121=0.0009762580199220349, P122=1.7247316772311472e-05, P123=1.1131333908432882e-05, P124=5.255305048678245e-06, P125=7.76985237637885e-05, P126=4.039834100987452e-05, P127=1.0546105741111751e-05, P128=8.290680722218289e-06, P129=2.3412479827925553e-05, P130=2.0401429317639077e-05, P131=1.561496525200067e-05, P132=1.4426930439699986e-05, P133=2.2858771494011478e-05, P134=6.428926486609569e-06, P135=6.85187112339136e-06, P136=1.1469262041766717e-05, P137=1.0213371704369069e-05, P138=1.0611870932370806e-05, P139=1.1056403743620077e-05, P140=1.1417797832172343e-05, P141=1.5909801298200704e-05, P142=3.1563466368296773e-05, P143=2.0931526147318957e-05, P144=2.4934866085940303e-05, P145=1.542499511091483e-05, P146=2.007512049516259e-05, P147=1.955060496718645e-05, P148=1.7632326144159643e-05, P149=2.8407601292336395e-05, P150=2.276643213466085e-05, P151=1.693146131175077e-05, P152=1.292863049016922e-05, P153=2.2413971982652216e-05, P154=1.7325674873276902e-05, P155=1.724547575682249e-05, P156=2.1040948744712636e-05, P157=2.4933243363644957e-05, P158=2.2933111217098098e-05, P159=2.9725522172031198e-05, P160=3.218419559234874e-05, P161=5.223478059608843e-05, P162=4.6450852255477654e-05, P163=4.034652273807136e-05, P164=4.484236524576703e-05, P165=4.689655072938109e-05, P166=3.898004000453648e-05, P167=2.6104352674414872e-05, P168=3.436373332043094e-05, P169=2.947710945204232e-05, P170=2.489458413215966e-05, P171=2.2163158803072276e-05, P172=2.557033890078855e-05, P173=3.0616596366433056e-05, P174=1.8449646192537575e-05, P175=1.9401057995289546e-05, P176=1.9553736429247455e-05, P177=1.4877119348473423e-05, P178=9.838615242081264e-06, P179=1.0374291490073992e-06),Row( P0=-803.931252254351, P1=41.26090716990407, P2=-5.088422087548676, P3=-8.083700317347436, P4=-15.292203683389728, P5=-6.11198505573065, P6=-6.5104181972285335, P7=-6.286768993146657, P8=-6.324841368597613, P9=-9.799078652396227, P10=-6.691923737495635, P11=-4.102033235143375, P12=-4.0684139456283726, P13=-5.668708614009538, P14=-1.196000036728797, P15=-2.767202217759659, P16=-2.9006152988133516, P17=0.4572792520752151, P18=-1.0838568919106828, P19=3.3863314054233773, P20=4.3662321435295155, P21=5.223040296548847, P22=2.876310610614351, P23=3.001686315116725, P24=-1.4990178488965156, P25=-0.0686866906661114, P26=-2.8645239760035337, P27=-0.9590856877507812, P28=-1.179962782077708, P29=0.8477866200118478, P30=-2.4913882317406295, P31=-0.47292170114252385, P32=-3.868487833105216, P33=-2.021456128193695, P34=-3.5851303485801087, P35=-2.801047442874024, P36=-3.055216326314677, P37=-1.8846027952050945, P38=-2.5163518461568484, P39=-0.09232315017790402, P40=0.5162133664910703, P41=0.5234282836622641, P42=0.5189405090745944, P43=0.49431854000726516, P44=0.5183309190704488, P45=0.5514068092387066, P46=0.6249637079520054, P47=0.6095302489274159, P48=0.5661944136587297, P49=0.5329231812656301, P50=0.5187178689851051, P51=0.4927709565319247, P52=1.0081510836795667e-07, P53=1.3160135113865404e-07, P54=2.6418075808064386e-07, P55=9.637389926930745e-07, P56=2.462738790511502e-06, P57=2.0902999214472518e-06, P58=9.359849481723933e-07, P59=1.2198082723279338e-06, P60=3.9126084573517636e-06, P61=3.4341118836467266e-05, P62=0.0005204178691490172, P63=0.001112326518476043, P64=0.0007235987820101311, P65=0.0007453287996070883, P66=0.001023729126359329, P67=0.0010149704262982864, P68=0.0009401669618248398, P69=0.001894881988058933, P70=0.0027834728064882234, P71=0.0013243118019257577, P72=0.00019747368505246613, P73=9.099536176819105e-05, P74=0.0001328116348630981, P75=0.00020628623179068275, P76=0.0001653151090621564, P77=0.0002721587969787687, P78=0.00028942021898947817, P79=0.00028617151392268267, P80=0.0003544348409955843, P81=0.0003948723302387161, P82=0.00037239335885258626, P83=0.00045269372823471363, P84=0.0009070360145940807, P85=0.001385745065619432, P86=0.002178022487792069, P87=0.0031511332983311683, P88=0.0030206969583148613, P89=0.001522737040703367, P90=0.00038296784696827433, P91=4.559363570803655e-05, P92=1.8338593190122676e-05, P93=1.176068301604946e-05, P94=2.6388381237084062e-05, P95=4.025135509465625e-05, P96=2.2910084968336687e-05, P97=4.940856023186364e-05, P98=6.178386342145044e-05, P99=1.886781146899347e-05, P100=2.2979922595638027e-05, P101=2.5041452717058892e-05, P102=2.0823196540914918e-05, P103=2.3844196728128868e-05, P104=2.56659776208892e-05, P105=4.7538184835259014e-05, P106=3.7340668211799734e-05, P107=2.6571279605682198e-05, P108=2.4260632667131644e-05, P109=8.114977426296593e-06, P110=5.007935634839398e-06, P111=5.882350914714698e-06, P112=8.346067115008064e-06, P113=4.540051289974271e-06, P114=7.915432512059484e-06, P115=1.3085245253945029e-05, P116=1.0796915031241397e-05, P117=1.1239943954343155e-05, P118=5.149048898748581e-06, P119=2.6945619711266943e-06, P120=3.8040760713299497e-06, P121=4.604215538683092e-06, P122=3.4578495263492033e-06, P123=2.517724814375601e-06, P124=1.4623042068594206e-06, P125=2.3912624279374893e-06, P126=2.0987202826155355e-06, P127=1.2591916412177883e-06, P128=2.09254900741621e-06, P129=1.4075183429235316e-06, P130=8.262469787934493e-07, P131=1.277490799559838e-06, P132=8.001131545739418e-07, P133=1.114363389766422e-06, P134=1.7141245909270861e-06, P135=2.9974465116374653e-06, P136=3.655195151921108e-06, P137=3.662581077115226e-06, P138=2.899522999457744e-06, P139=1.6710639197340253e-06, P140=1.6047164722974264e-06, P141=2.3881658010081453e-06, P142=1.965007147451858e-06, P143=4.674118435648524e-06, P144=1.232843575579217e-05, P145=1.0589392772271992e-05, P146=3.911128162808272e-06, P147=1.3357766613981738e-06, P148=1.905734265997056e-06, P149=2.278235403553414e-06, P150=3.931639572488735e-06, P151=4.3159574335263945e-06, P152=1.94377249860246e-06, P153=1.0337274532072187e-06, P154=8.507129167964854e-07, P155=1.103248543654817e-06, P156=1.4421667442632913e-06, P157=1.947915427810982e-06, P158=4.810771649357826e-06, P159=4.394259875566851e-06, P160=2.7588147791077478e-06, P161=3.974195634047343e-06, P162=2.7753721381399384e-06, P163=4.898986389957291e-06, P164=6.021966954144183e-06, P165=8.08008401515213e-06, P166=5.098715842989155e-06, P167=5.792644975540707e-06, P168=5.119246876151872e-06, P169=6.811051680531316e-06, P170=4.344286995053472e-06, P171=6.873787013542933e-06, P172=6.288060887482295e-06, P173=1.1286406626916863e-05, P174=7.854804428547423e-06, P175=4.2552939636551435e-06, P176=2.569007945339095e-06, P177=2.504056877166581e-06, P178=1.7245922327257228e-06, P179=2.545184632317821e-07),Row( P0=-675.6090723219153, P1=35.04718464292281, P2=-21.090526580786673, P3=-15.180318980495954, P4=-19.69628288410732, P5=-10.982052038862461, P6=-6.695500161108398, P7=-3.963974236161833, P8=-4.146849092763271, P9=-9.175023288109017, P10=-4.7661662854055376, P11=-7.168611433790643, P12=-2.1950094919178924, P13=-2.6007502263503297, P14=-1.5796922306333365, P15=5.362099439096234, P16=1.3904321939012787, P17=6.486366134871197, P18=4.823057131871033, P19=1.842634393313414, P20=-2.651148699573085, P21=-1.4871172279955471, P22=-3.3155792922852387, P23=-0.951191248168682, P24=-2.3698529606494634, P25=-1.3876558096255902, P26=-4.824384043498208, P27=-1.6806244954286838, P28=-6.044122234392616, P29=1.2647214712010415, P30=-0.3858461053254219, P31=0.8494967122346386, P32=-2.383385670065011, P33=1.1695579720969482, P34=-0.5981248653864762, P35=0.9968286306700435, P36=-3.1023932102838034, P37=-0.4639687733754075, P38=-2.21619768620112, P39=0.4264286763554254, P40=0.606631823625632, P41=0.570564478488078, P42=0.5282436574340227, P43=0.5068752628394854, P44=0.5232608270581673, P45=0.5310167750332141, P46=0.5095398935502837, P47=0.5318690510328172, P48=0.5317652481656854, P49=0.5302616866684843, P50=0.5536394850044114, P51=0.5847823957857042, P52=4.6375781175804326e-07, P53=1.0264180351879838e-06, P54=3.586777651676138e-06, P55=1.0482126781755457e-05, P56=1.912298502342532e-05, P57=2.5011976941358334e-05, P58=1.8368531493277352e-05, P59=1.4892244743340327e-05, P60=1.7134134009038504e-05, P61=3.5162713210038276e-05, P62=0.00011056500762227619, P63=0.0004076301887531903, P64=0.001104396626066308, P65=0.004286807171276242, P66=0.011089349035001222, P67=0.019167133665124205, P68=0.030086547254100753, P69=0.028746960731204394, P70=0.011310530074138068, P71=0.005333066075645579, P72=0.004589300237659781, P73=0.0019354851427065643, P74=0.0004770653819154331, P75=0.0008795151734169118, P76=0.0010955270036051964, P77=0.0011094563936245624, P78=0.0018617844999479606, P79=0.003890898202862493, P80=0.010269133898943109, P81=0.014818475203269083, P82=0.016355827925747635, P83=0.03217509874708496, P84=0.06426834129500279, P85=0.03884652693359128, P86=0.009617340978241327, P87=0.01175990778716193, P88=0.010903886305924168, P89=0.005910090961746152, P90=0.01191755730354928, P91=0.025342412121556877, P92=0.026889990407493736, P93=0.03835376552276816, P94=0.034919363302902844, P95=0.02962548433661296, P96=0.03561306306500839, P97=0.019442275801771772, P98=0.0023679887582677933, P99=0.0015908075574452476, P100=0.0009758784706339601, P101=0.0013702976125322297, P102=0.002421167517138659, P103=0.004772436222847109, P104=0.002679755182054571, P105=0.0019096246930900196, P106=0.0014852311505328458, P107=0.0006961420597788729, P108=0.0008889152716779674, P109=0.0017877988928788828, P110=0.0016286801295753825, P111=0.0012546352670410443, P112=0.0016036714627406085, P113=0.002970555620307687, P114=0.0013536173246012511, P115=0.0004937199526577615, P116=0.0003977916803071325, P117=0.0007376624722301657, P118=0.0011118241476477417, P119=0.001655054614045744, P120=0.0023952549617391595, P121=0.002071289923729491, P122=0.0010750350872805261, P123=0.00030737567906678124, P124=0.00020386078129120376, P125=0.00014042114068304546, P126=0.00014209665760760127, P127=0.0001329817680669971, P128=0.00015212292293747433, P129=0.0001138738797571647, P130=0.00011782451802196166, P131=4.680127237224929e-05, P132=9.731646191244092e-05, P133=0.00017312006589961546, P134=7.643075254459581e-05, P135=0.0001276485004755508, P136=0.00014428655550203823, P137=0.00018313470760927, P138=0.00015417361504588606, P139=8.350148477611038e-05, P140=7.873926576305474e-05, P141=6.081097044556068e-05, P142=7.000825220593284e-05, P143=5.0451051531015724e-05, P144=5.6103331126885366e-05, P145=6.766654698766276e-05, P146=6.795275014176412e-05, P147=3.9159902939107234e-05, P148=3.498540631585652e-05, P149=2.8448629815649956e-05, P150=2.550104870313048e-05, P151=2.8172960142423437e-05, P152=4.010485289812768e-05, P153=3.6198207363692245e-05, P154=2.6841379705285036e-05, P155=4.641160279299736e-05, P156=7.481062978931764e-05, P157=5.600233273860125e-05, P158=3.7570054440419296e-05, P159=2.013203160740951e-05, P160=2.721916551723501e-05, P161=2.089598762997428e-05, P162=2.079926820929569e-05, P163=2.402908381119775e-05, P164=3.430143841710676e-05, P165=4.2676594934408855e-05, P166=4.852804586986994e-05, P167=5.058402394274863e-05, P168=5.331352754638194e-05, P169=3.604450771141932e-05, P170=1.929964958277498e-05, P171=1.0090599657997518e-05, P172=8.088501104441024e-06, P173=1.4535551054720667e-05, P174=2.335898327517379e-05, P175=3.222132746337458e-05, P176=4.460065935962498e-05, P177=5.568713009662778e-05, P178=1.5814332186533405e-05, P179=6.716713638575502e-07),Row( P0=-482.35217583446195, P1=-9.692096742652192, P2=-44.49436171360949, P3=-9.476328285401525, P4=-21.56052750754989, P5=-4.041327600820808, P6=5.763013167310037, P7=5.717693725161763, P8=13.567974150794354, P9=-6.450562586757335, P10=-7.358219403119574, P11=-8.56967837690413, P12=-5.6363768397497385, P13=10.097514900157385, P14=0.40035958570017594, P15=-1.0800987963941224, P16=-11.442535721977853, P17=1.6974008472302067, P18=2.3063174921736, P19=1.9286780260961094, P20=-0.6859434998065227, P21=4.733450861723682, P22=-1.5269072285438592, P23=-3.10783395110604, P24=-3.9143051417392587, P25=3.6509038258507758, P26=2.6131948338502875, P27=-2.4685759326349137, P28=-1.281568271931045, P29=2.0429644629127623, P30=-2.9364602322575495, P31=-0.09394435129342692, P32=0.6952486279751856, P33=-2.3008550666387397, P34=-2.0281893462190106, P35=0.923449732816034, P36=-2.3700870321139815, P37=-0.8750948778991092, P38=0.14142466688866323, P39=1.4022402171893549, P40=0.46013867597507285, P41=0.4784294131971488, P42=0.5341832557569756, P43=0.5516991659409526, P44=0.6512780990396504, P45=0.5338403291159234, P46=0.4181124753952017, P47=0.36965158571601997, P48=0.3633258618695494, P49=0.39502700871842955, P50=0.4585422465701125, P51=0.4800458088060409, P52=5.865917843376413e-06, P53=1.1416984638610018e-05, P54=1.660132327162662e-05, P55=2.0295470754996445e-05, P56=3.183535795241091e-05, P57=5.461959998379218e-05, P58=7.200203137940705e-05, P59=8.545697215751531e-05, P60=0.00013355210399223165, P61=0.0004373053089513405, P62=0.0007956353032592662, P63=0.0006047754747795376, P64=0.00031242558138010364, P65=0.0003046670119005672, P66=0.0005672259964750831, P67=0.00043098777967793433, P68=0.0001649580542379163, P69=0.00024293723066234177, P70=0.00042145455531955226, P71=0.00036970244247518184, P72=0.0005478588957070322, P73=0.0006526453559147126, P74=0.0010525906827634083, P75=0.0014057043095231131, P76=0.0016898954929364211, P77=0.005460767673308371, P78=0.008731441148600033, P79=0.01093231853682179, P80=0.017661086056520555, P81=0.024206281235697562, P82=0.03036486366479691, P83=0.03782631503962192, P84=0.06890107256205766, P85=0.11377065599798977, P86=0.2670681122949981, P87=0.5296888902979846, P88=0.8714235982267847, P89=1.4043551972271349, P90=2.158676045733881, P91=4.7274145134081245, P92=3.2752644362717787, P93=1.0888074604848323, P94=0.8645285263362564, P95=0.3709550331714468, P96=0.06295469944914991, P97=0.06433588109207837, P98=0.05645825108736398, P99=0.061889475088275796, P100=0.09994427811565305, P101=0.07996604630524697, P102=0.37432703565656705, P103=0.8939301613308738, P104=0.3239527154075245, P105=0.08646607000577643, P106=0.07752896458161714, P107=0.20799751805871466, P108=1.2898743601807459, P109=2.8800369574249594, P110=2.9465399153843417, P111=1.489656497075913, P112=4.254725669971706, P113=2.5210491552665, P114=0.1520986862293787, P115=0.3819479850899511, P116=0.33592223453155906, P117=0.09446110284475825, P118=0.1020407502090482, P119=0.10570397148320794, P120=0.11689190800565635, P121=0.21702612138195665, P122=0.7485371272614602, P123=0.8698604237935815, P124=0.3537284184653669, P125=0.4060582661004527, P126=0.09294838081058572, P127=0.019441653359575584, P128=0.029963859815881802, P129=0.022602414610683477, P130=0.03386455763236666, P131=0.06351846257470045, P132=0.06784923220262333, P133=0.041001081634579974, P134=0.019690280722320475, P135=0.011126954639522053, P136=0.02339477618739068, P137=0.017018584039428384, P138=0.08228219147206069, P139=0.11380641104302937, P140=0.051394447805097686, P141=0.022423958741701024, P142=0.01062701930017052, P143=0.013906072808538833, P144=0.014626947239450986, P145=0.012379043048925582, P146=0.014167729432366202, P147=0.01042289376976054, P148=0.030920740788980012, P149=0.04396797786564097, P150=0.0275130357696376, P151=0.01764710415442945, P152=0.021847930074419826, P153=0.011140429512581594, P154=0.009764516210669807, P155=0.00328397132539427, P156=0.0031617448371440604, P157=0.0023105845800936975, P158=0.0010525321156048023, P159=0.0008281863892118716, P160=0.0005833383716822135, P161=0.0005975525230017355, P162=0.00028491578161962135, P163=0.0003129918251639158, P164=0.0003042438030020903, P165=0.0003380958834694553, P166=0.00025556218379898165, P167=0.0002523907680633014, P168=0.00034338642813263334, P169=0.00046022024824582636, P170=0.0007034600982864386, P171=0.000671024587823602, P172=0.0005349241924465286, P173=0.0005684124750644241, P174=0.0005381332487320166, P175=0.00041846890580895835, P176=0.00039505515383897596, P177=0.0003340867948068294, P178=0.00015607148333952604, P179=2.4691101526206216e-05),Row( P0=-628.8835339219305, P1=32.109521113420186, P2=-13.920114891215801, P3=-14.798742689866053, P4=-24.35357532867473, P5=-11.89228115427878, P6=-9.935486408123863, P7=-4.597984723389141, P8=-9.219706986313508, P9=-11.028688729470925, P10=-6.686840491833167, P11=-7.85064439364664, P12=-5.589030409948991, P13=-1.862454432171508, P14=-0.5340219535255716, P15=2.2657721219383355, P16=0.2590270338147828, P17=0.7926119498488432, P18=0.24915135402723387, P19=-0.5009423830239166, P20=1.2838236962795522, P21=0.16048727027041923, P22=-4.4548294440984195, P23=-2.037301934368927, P24=-7.061343985120716, P25=-3.421492035799764, P26=-4.156261710330891, P27=-0.9581862133388483, P28=-3.8790190837864613, P29=0.20984879074082313, P30=-1.5612342627439124, P31=0.8785651163572019, P32=-0.9744920424326288, P33=-0.14933326377575545, P34=-3.217494286359385, P35=-1.290799440635517, P36=-1.5456047797972774, P37=1.4199708292693245, P38=-1.113971354807365, P39=-0.1348538305556989, P40=0.5958949782843376, P41=0.6407872724552762, P42=0.669943450324114, P43=0.6057982784567533, P44=0.5654475262614714, P45=0.5284072943553787, P46=0.5057475657207399, P47=0.514116424163889, P48=0.5486066036776376, P49=0.6049160183192491, P50=0.5890796745436436, P51=0.5647172899229633, P52=1.2003507491443597e-07, P53=4.3925853550968197e-07, P54=4.804388145036812e-06, P55=1.4243908038519527e-05, P56=2.2118035930821697e-05, P57=3.4497751163565545e-05, P58=2.4716650679082472e-05, P59=2.472708408570408e-05, P60=3.693832486423199e-05, P61=8.002938704494034e-05, P62=0.00035829721083308595, P63=0.0010708411805733901, P64=0.0030491119935307197, P65=0.01138959952520286, P66=0.019990135092905505, P67=0.02355864682574372, P68=0.039108268571160294, P69=0.09512544504869258, P70=0.12354200114733388, P71=0.03192447533960769, P72=0.004371884642704593, P73=0.0041479591634307564, P74=0.009163682494115734, P75=0.009427525507839768, P76=0.003626907052885802, P77=0.0024810535429597266, P78=0.003442062697065238, P79=0.00631024356556319, P80=0.0075764706065608155, P81=0.005286912572510916, P82=0.006282216110711284, P83=0.013891152469519894, P84=0.035976494245230264, P85=0.08375752622134067, P86=0.07714654644214895, P87=0.06428361504915928, P88=0.0802092042694073, P89=0.03874432799908971, P90=0.013803207904875443, P91=0.009881508446985971, P92=0.005297016355165174, P93=0.0021335461899330706, P94=0.002068847249811211, P95=0.0016464122857466396, P96=0.0010584816347708032, P97=0.002122619242079383, P98=0.00305549122056862, P99=0.002551801251586762, P100=0.008908978483111929, P101=0.006889787934384223, P102=0.0027269311290161117, P103=0.0019611310114612725, P104=0.0020573000127038126, P105=0.0021180663229565144, P106=0.001939217257537399, P107=0.00195456481006874, P108=0.0010282190361675637, P109=0.00030956803832822365, P110=0.0002263439095835022, P111=0.00022100621255865112, P112=0.0003346301838839659, P113=0.0004081079838269343, P114=0.00019220494637488727, P115=0.00017099128097649958, P116=0.00019125278635360795, P117=0.00045758182705422733, P118=0.0007909468639686294, P119=0.000470442790734216, P120=0.00023363038758548546, P121=0.00024213993181773635, P122=0.00018257903536535503, P123=0.00011010737392300075, P124=9.710029173381086e-05, P125=0.0003359749365136724, P126=0.00015854413138449635, P127=0.00010831427574194609, P128=0.00022373564791175693, P129=0.00027064369176917194, P130=0.0005561581187810355, P131=0.0005118450289918307, P132=0.0001542421711081628, P133=0.0001689546116776148, P134=0.00022828521945642206, P135=0.00011678810162771671, P136=8.943551728526378e-05, P137=0.00011054643441718771, P138=9.504169526168247e-05, P139=9.900313991950651e-05, P140=0.0001322051137625659, P141=0.00013319898657540384, P142=8.526269798409837e-05, P143=5.952057820580221e-05, P144=4.228842489699764e-05, P145=3.071525463531077e-05, P146=3.555015064684845e-05, P147=4.642384781419821e-05, P148=7.012177726642066e-05, P149=7.199927587214976e-05, P150=0.0001325464757513254, P151=0.00015546565472093407, P152=0.00017960381245658307, P153=0.00024588501989472475, P154=0.00013482876189770143, P155=0.00011730264201457457, P156=0.0001567461179294343, P157=0.00011051787295865798, P158=7.652787162272103e-05, P159=9.20256369370471e-05, P160=0.00012841489585524975, P161=0.00021868441997226028, P162=0.00018874671593741742, P163=0.00026850934115999316, P164=0.0003632283406159204, P165=0.0002952740483698259, P166=0.0003713996390695223, P167=0.00038573101555661616, P168=0.0003391180402829778, P169=0.00031895791291189205, P170=0.00023772474335221233, P171=0.0002496436340217726, P172=0.00040089072974219136, P173=0.00035615530962620107, P174=0.00037479874403672066, P175=0.0003022978785120167, P176=0.00019510969893250292, P177=0.00023126840966793062, P178=0.00015317674860622447, P179=1.2780506468480127e-05),Row( P0=-523.4331393760199, P1=-5.965463124901217, P2=-43.81615181287808, P3=-14.43760434076277, P4=-26.104374793370237, P5=-11.568399482251165, P6=1.848813692999181, P7=3.5796464681050284, P8=15.487560927720937, P9=1.6874803281580784, P10=5.897167985600803, P11=-0.9011333163321509, P12=-13.345238343560116, P13=-0.2873043441458996, P14=-1.227592496965964, P15=3.4919868150278672, P16=-4.448056382446711, P17=-1.6300399520310052, P18=0.4041409788426925, P19=2.3527609611741713, P20=1.2241089294121876, P21=0.6014235556778413, P22=-2.555236858782035, P23=2.0077152214047307, P24=-1.3768680880068236, P25=-2.2221018020672956, P26=-0.12659944911194543, P27=-0.5001034782077873, P28=-1.8617411902082281, P29=1.639680281250571, P30=-1.0275683274065484, P31=-0.07453035799451299, P32=0.2676519567964049, P33=-0.05979092497028226, P34=-1.9002012732960236, P35=-0.35713219711476313, P36=-0.12344302912833695, P37=1.130727025167728, P38=-2.1028203619731496, P39=1.3734676260033345, P40=0.5688647191897451, P41=0.5768247707971483, P42=0.5733020333250781, P43=0.47656949303431, P44=0.4281474181292281, P45=0.40932500894245044, P46=0.42403261943776804, P47=0.41691776635455247, P48=0.41465491917143293, P49=0.507403912716724, P50=0.5136397472514869, P51=0.5288835582325032, P52=1.3729179154664927e-06, P53=3.229483992396659e-06, P54=5.118647195878598e-06, P55=5.7242645400791e-06, P56=8.830474986124024e-06, P57=1.5002798066122254e-05, P58=1.5573615073364848e-05, P59=1.801816520023199e-05, P60=2.3771050960767934e-05, P61=2.6043452515896876e-05, P62=2.750719908137258e-05, P63=2.6148526241096907e-05, P64=1.7100442320772673e-05, P65=2.1430197230327956e-05, P66=2.718522846911088e-05, P67=2.5142860483468625e-05, P68=2.027178847645558e-05, P69=2.543405995233473e-05, P70=2.1571737167109717e-05, P71=0.0001361444687994191, P72=0.0007955763715479607, P73=0.001582245709537536, P74=0.003370153034505859, P75=0.0037275240608141366, P76=0.003046624272681722, P77=0.009018330496466784, P78=0.029511614149278824, P79=0.05289738252675497, P80=0.06762195506813032, P81=0.0972464570405589, P82=0.15352151296133917, P83=0.23527601426922834, P84=0.36570946848159863, P85=0.4033654802645868, P86=0.8598197788770142, P87=2.0178130121645848, P88=1.592737808740636, P89=0.6720899059691535, P90=0.25853885157902906, P91=0.14972371985168637, P92=0.131125667630071, P93=0.06791798369919502, P94=0.08999658494017862, P95=0.09214920953555455, P96=0.04607464621802362, P97=0.06786013561962635, P98=0.16429433038360616, P99=0.4092736238089447, P100=0.44550522979991325, P101=0.22525157579375854, P102=0.11422832670855049, P103=0.3341067193154574, P104=1.0081034415784136, P105=0.6124375381488142, P106=0.23532712998729954, P107=0.12524420543951154, P108=0.1814413151932551, P109=0.12090209165053525, P110=0.12781406230677414, P111=0.07303416592199351, P112=0.035533887475841394, P113=0.025271179917574167, P114=0.023532057932349137, P115=0.04992204809892429, P116=0.16728033356346528, P117=1.3528900704041957, P118=0.47463578537889334, P119=0.02497086802506644, P120=0.0784825336548837, P121=0.09182943704824817, P122=0.0977017696127219, P123=0.06991506840291223, P124=0.04169293222119346, P125=0.05838774015024943, P126=0.028376129865066695, P127=0.030751463124279236, P128=0.03630572986788043, P129=0.0727963087055795, P130=0.09842194331518725, P131=0.016459342768006483, P132=0.022057204741801454, P133=0.021606142057779006, P134=0.007276004755942694, P135=0.008490374959070292, P136=0.007876423301498355, P137=0.012751171482846043, P138=0.02966763280582878, P139=0.02702747481268932, P140=0.00903984689868654, P141=0.012077134855768893, P142=0.013412189617865314, P143=0.0038586674016433046, P144=0.004304905690446303, P145=0.0030171463538104996, P146=0.004475893077760708, P147=0.008124973161032138, P148=0.012975048288593157, P149=0.017834020155443223, P150=0.015536659423276849, P151=0.017839349704886532, P152=0.005983742584725903, P153=0.004424224398878008, P154=0.0035387219349605433, P155=0.001550640614140477, P156=0.0005875200860687319, P157=0.0004932970928696611, P158=0.0004090758264703558, P159=0.00041490581659111485, P160=0.0005013367401894851, P161=0.0007445837226279831, P162=0.0006065934078079316, P163=0.0004437004283268689, P164=0.0005516618652023858, P165=0.0005329649487279315, P166=0.0004900857344265116, P167=0.0005205439175920999, P168=0.0005034256765572761, P169=0.0005602390233551403, P170=0.0008615325497360601, P171=0.0006026467471841926, P172=0.0005105788268289775, P173=0.000560473539369553, P174=0.0009015348147488413, P175=0.0010072772330001422, P176=0.0006159523270092191, P177=0.0005124623495894017, P178=0.0002836490673106481, P179=3.209942314340965e-05),Row( P0=-490.4012463299109, P1=2.569267833540327, P2=-43.51838651244048, P3=-8.734376670547181, P4=-24.06355515048839, P5=-8.668594497660001, P6=6.454544882095516, P7=1.1272751693015124, P8=16.091106157930902, P9=7.121395613325119, P10=5.648593772677349, P11=0.28562731452582907, P12=-11.021029850693814, P13=4.029971848335313, P14=-3.9357459611210777, P15=7.1079601969780395, P16=-1.233041384343869, P17=1.283330523296965, P18=-0.023680074145773826, P19=0.7669937827786816, P20=1.6880359708334616, P21=1.1941095100122567, P22=-0.27139159200192225, P23=2.4744963104176896, P24=-1.7661889106436126, P25=-2.4633432245990035, P26=-0.7322746135518504, P27=0.17756001359151993, P28=-0.5726903284292073, P29=1.3432672397295404, P30=-1.263621029334388, P31=1.512403890045118, P32=-0.4569158219314987, P33=0.25323121947447186, P34=-1.154179626848699, P35=0.030741382072768407, P36=-1.4108855037263606, P37=-0.1462305077876733, P38=-0.29428036512618544, P39=0.8565454125604465, P40=0.584935747360563, P41=0.6021102515213372, P42=0.6121067457496012, P43=0.5445407360232076, P44=0.47823722438594346, P45=0.43191875014981507, P46=0.4278523432087851, P47=0.4493048991633347, P48=0.4834651674384697, P49=0.4943790089444228, P50=0.492574941162989, P51=0.5347344165167315, P52=0.002278972727284876, P53=0.001323964200879724, P54=0.00044886589981369643, P55=0.0002351951374090675, P56=0.00019681658204497395, P57=0.0002334667555836312, P58=0.00015113781374924998, P59=0.00011084143782359607, P60=9.089376570651723e-05, P61=0.00011312290652123066, P62=0.00015461653633841162, P63=0.00012654544704241525, P64=7.108061620194204e-05, P65=9.13176216515256e-05, P66=8.955838763829748e-05, P67=0.00012727068610996636, P68=0.0004484424080231027, P69=0.0011691108671442448, P70=0.0014777988661584453, P71=0.0035625073764926868, P72=0.006347420236766348, P73=0.006397532376899752, P74=0.011696847898560774, P75=0.019869177700832834, P76=0.03058743447620078, P77=0.061561291097466685, P78=0.07696500505551568, P79=0.09908635982864311, P80=0.12878309733160503, P81=0.20524822034167436, P82=0.26839495872510166, P83=0.4331519746709943, P84=0.9732564251986743, P85=0.9951606372874087, P86=1.0201679820428098, P87=1.396619679609777, P88=1.6247302632856055, P89=1.5378789176554282, P90=1.8540862602861996, P91=0.7268536131704794, P92=0.05819491292437628, P93=0.012962580137907265, P94=0.022645477567789004, P95=0.025005771189894752, P96=0.019898620670212724, P97=0.1492836089897526, P98=0.2319036629994535, P99=0.2354319857338812, P100=0.4659652954393222, P101=0.7109782406486819, P102=0.6515223335477635, P103=0.5494564706768252, P104=0.22674932102332676, P105=0.20455529005806372, P106=0.10547216082644761, P107=0.10633655354364584, P108=0.17694938860278125, P109=0.1440033603735144, P110=0.18848418348334567, P111=0.13324994193566458, P112=0.2817616061960202, P113=1.2446496947231454, P114=0.5723032138164551, P115=0.28031488703714996, P116=0.2676280081398275, P117=0.3739732517378402, P118=0.15116530804024772, P119=0.10130342881780006, P120=0.13956759202102997, P121=0.0775808226822332, P122=0.02836777369662849, P123=0.0299441744692924, P124=0.060446088998749745, P125=0.07700483094900366, P126=0.08771419020372459, P127=0.1033692682581895, P128=0.030842165742163676, P129=0.06099011956317593, P130=0.07999214359803596, P131=0.06790692881371359, P132=0.12840232615099995, P133=0.16631098953447418, P134=0.024984154616457317, P135=0.014437500983539613, P136=0.026923529121719116, P137=0.025771731285615725, P138=0.04550731224150031, P139=0.05984749850641246, P140=0.02743903039942376, P141=0.020959458039210403, P142=0.01456413495899745, P143=0.015211027938907232, P144=0.012180019609810603, P145=0.012995673327037905, P146=0.01462635259819809, P147=0.01607374197904927, P148=0.018977426949444386, P149=0.029362144141334312, P150=0.03258448068927717, P151=0.026095228915793203, P152=0.010646983847176212, P153=0.008091919893808591, P154=0.004147044287179428, P155=0.0025413994416354836, P156=0.0008882584858014068, P157=0.0005409444088474606, P158=0.000553262181192436, P159=0.0005737047698243797, P160=0.00045967730205937504, P161=0.0004398625832135894, P162=0.00042132961424030854, P163=0.0004640918166705524, P164=0.0006228959320423028, P165=0.0006741016879754463, P166=0.0006013868514495065, P167=0.0004793334459983317, P168=0.0006573131692487169, P169=0.0006188414329789993, P170=0.0005443760463567365, P171=0.0005549322259474422, P172=0.0005019719395115323, P173=0.0004676582800793219, P174=0.0003821278113772107, P175=0.0003767679942726985, P176=0.000410496605476098, P177=0.0003575722199963471, P178=0.00019692548094990176, P179=1.4830860250558014e-05),Row( P0=-628.0904497654035, P1=36.876060202116435, P2=-26.953974977766656, P3=3.0260457539111227, P4=-20.79158902846924, P5=-8.92080126926604, P6=-7.636051474501071, P7=-6.990319304678941, P8=-1.572999607595394, P9=-8.342224421785193, P10=-1.481639297965876, P11=-1.048193037726238, P12=-3.190988232787613, P13=-0.5674427441043015, P14=-4.38010084282619, P15=3.143130281419846, P16=-0.5330755780010987, P17=7.239191288375873, P18=6.844798829013362, P19=4.233231397984713, P20=2.3451207209661953, P21=0.39197883469545314, P22=-2.5870353162142203, P23=0.365917740851368, P24=-3.9841159548887233, P25=-2.655145147622412, P26=-1.56487221816447, P27=-2.7249343362414424, P28=-2.3214191939958146, P29=-0.631092880411015, P30=0.3448545648069935, P31=-0.9914234198348219, P32=-0.905579910970355, P33=0.39594287368232034, P34=-0.7371265153421329, P35=-0.8061027839841923, P36=-2.405579128214055, P37=2.711528778529538, P38=1.231138302945104, P39=0.4361708995387913, P40=0.6529422761176389, P41=0.6404856539908443, P42=0.6263708860381485, P43=0.6020901774254317, P44=0.5941079371320654, P45=0.6048365061306658, P46=0.5997216129438301, P47=0.6193684326105693, P48=0.7014489710756584, P49=0.7537564556732711, P50=0.6844216280041602, P51=0.632172390967335, P52=4.166017640567223e-06, P53=7.31397333965868e-06, P54=1.3182846833091179e-05, P55=2.0124060406476996e-05, P56=5.25373874556229e-05, P57=0.0001555370039254353, P58=8.442280836567609e-05, P59=9.744555540866466e-05, P60=0.00025367615627851946, P61=0.00019631589173916723, P62=0.0007546724795182661, P63=0.0033772332251651056, P64=0.01000635453842562, P65=0.03116550481478971, P66=0.04956048650368121, P67=0.07522256471600712, P68=0.07349484408644116, P69=0.014385584236560897, P70=0.005226260659843098, P71=0.007889096912810511, P72=0.02356652029348296, P73=0.04798942745249737, P74=0.024766360616449788, P75=0.008119300530518356, P76=0.016763459692442965, P77=0.0845750617757643, P78=0.12142141531637915, P79=0.07029329865396386, P80=0.04073441022910644, P81=0.018636540575547023, P82=0.00975036341908868, P83=0.013339385969086768, P84=0.03537293987336519, P85=0.018871109952595687, P86=0.005283555671216462, P87=0.004183577577155757, P88=0.0036402647682342026, P89=0.0034466040377161187, P90=0.004197412979827988, P91=0.022038888256306958, P92=0.07545700605392773, P93=0.036097274416704336, P94=0.02814094227655354, P95=0.024955738006286414, P96=0.014118861689719934, P97=0.011420336925543551, P98=0.012643889417632642, P99=0.00578009439503215, P100=0.020522841418694066, P101=0.020723658394422452, P102=0.02018845461884714, P103=0.016073670890681617, P104=0.009409574863733451, P105=0.0044877508979217036, P106=0.009679048606723945, P107=0.03954007499459501, P108=0.017682485964168968, P109=0.050014239753964074, P110=0.05058027328105848, P111=0.019345859540130537, P112=0.021473499057996476, P113=0.014184728123190432, P114=0.013053064491577546, P115=0.0396718363194316, P116=0.015498000056067236, P117=0.013914008825031349, P118=0.014636340630059133, P119=0.023100609283069458, P120=0.023410592772132918, P121=0.02219019360698907, P122=0.014558662324140192, P123=0.0032116774258468873, P124=0.003380940758138243, P125=0.007579135654053859, P126=0.006642279707826028, P127=0.003360079765168773, P128=0.0016073577168070768, P129=0.003148773713087149, P130=0.0015076769713235959, P131=0.002540380799045201, P132=0.0027918332112289524, P133=0.0007581978132886683, P134=0.0007824231108667466, P135=0.0016531161505406466, P136=0.0016685047521561621, P137=0.0016260169182567418, P138=0.002178805638701738, P139=0.0041532414496301216, P140=0.002483286198813702, P141=0.005367335027265875, P142=0.005424186191496472, P143=0.0016551240142356019, P144=0.0005645776501329994, P145=0.0006663667391664273, P146=0.00036073396216760017, P147=0.0005866108901553544, P148=0.0006553504874626909, P149=0.0007241279387622541, P150=0.0010658532093621045, P151=0.0008781892302095133, P152=0.0019125318614437593, P153=0.0008782784283499091, P154=0.0005220585092224098, P155=0.0002513232064916736, P156=0.00014222537018811478, P157=5.0618248672652845e-05, P158=3.43771117527736e-05, P159=3.542629880292192e-05, P160=2.7897723117627816e-05, P161=2.145229795389682e-05, P162=1.6537472288811295e-05, P163=1.956934891659551e-05, P164=1.6524999416328002e-05, P165=1.556678018053744e-05, P166=1.1689085256619296e-05, P167=6.830149780483039e-06, P168=1.716918310835651e-05, P169=1.2869653974252117e-05, P170=2.679188626301698e-05, P171=3.1683084528609076e-05, P172=2.540801203655262e-05, P173=2.8402678099725205e-05, P174=2.517833351201961e-05, P175=2.350497430396089e-05, P176=1.4225026840646591e-05, P177=1.4486292090623029e-05, P178=6.009543592453827e-06, P179=8.447940841118364e-07),Row( P0=-596.0320932974205, P1=17.960115040732397, P2=-35.49798799926394, P3=-5.094903440288327, P4=-21.73003691068772, P5=-12.665822043426042, P6=-10.291464659970947, P7=-5.418297946165202, P8=-1.9190503521667375, P9=-7.484645843109601, P10=3.6549191726478485, P11=10.204538537811601, P12=7.809298082490951, P13=12.500266437835354, P14=5.261065594420222, P15=3.440186723494238, P16=-8.703831877471082, P17=-0.6841537057246595, P18=-2.913378824406768, P19=-0.8047061411459926, P20=-1.6801833796281043, P21=-1.8125625701174575, P22=-1.4226432833975604, P23=3.539371013454086, P24=1.4266552118654556, P25=-0.5021286896131664, P26=-1.1022899541807214, P27=-0.2645945042060375, P28=-1.2377525601165487, P29=1.5702689022671223, P30=-0.08936904102361815, P31=-2.9409383211042854, P32=0.6805135826095123, P33=-1.0684087915762615, P34=-1.1369497750860442, P35=-0.4284660510927349, P36=-0.07524413275807794, P37=3.233742555052603, P38=-1.678288825150377, P39=0.03881628129571682, P40=0.5979843835084117, P41=0.6067455369558696, P42=0.6045896043934598, P43=0.5561642080289985, P44=0.5906811641768234, P45=0.6106620064941618, P46=0.6212652126520873, P47=0.587394659154719, P48=0.5709449035546958, P49=0.6027818677835134, P50=0.5923059598470743, P51=0.55521558469044, P52=4.425976777414718e-06, P53=1.1574408365099438e-05, P54=1.884299423322083e-05, P55=1.9128091997883314e-05, P56=2.208462273847895e-05, P57=3.0117790816824423e-05, P58=2.356124560540513e-05, P59=1.4150989235179776e-05, P60=9.944412252855317e-06, P61=9.635914862400158e-06, P62=1.7140352573051475e-05, P63=2.5224551026666147e-05, P64=2.6045265076791157e-05, P65=5.509547098309847e-05, P66=9.294727533345435e-05, P67=0.00028673249094923693, P68=0.008839576328970032, P69=0.03989090079683377, P70=0.04050312546449529, P71=0.05954623052699988, P72=0.14727559679396857, P73=0.22370678200061253, P74=0.4423892793220981, P75=0.20224227409488219, P76=0.07858962287764287, P77=0.05398054988158346, P78=0.027745210826193967, P79=0.01344156669611016, P80=0.004990433301158968, P81=0.002063139403603808, P82=0.0012033122634102016, P83=0.0008767912663970206, P84=0.0009868888717374865, P85=0.0007730057137834137, P86=0.008495845649185478, P87=0.04290656289540078, P88=0.07629441604383098, P89=0.0741698260799992, P90=0.14512110924508714, P91=0.1411339210936677, P92=0.10261194317208948, P93=0.053257257195728415, P94=0.110611610813525, P95=0.13694333621705845, P96=0.045860895420667985, P97=0.008814943291095077, P98=0.003892997376009772, P99=0.028754901359241473, P100=0.07269260455065954, P101=0.0403146906214312, P102=0.008702199787461144, P103=0.010428213453098483, P104=0.07322504939482796, P105=0.11732198037636801, P106=0.09905587120759499, P107=0.051308659441659826, P108=0.018541230560311005, P109=0.01505865394265452, P110=0.013422416684540061, P111=0.0014488273625913556, P112=0.012868122977511512, P113=0.03896366060465229, P114=0.015514938079721095, P115=0.022285981720577757, P116=0.03576455194640817, P117=0.023565005975831464, P118=0.014914209985530894, P119=0.010924366285346687, P120=0.02325375420270463, P121=0.02552726080117823, P122=0.013978337276775719, P123=0.010306234971427004, P124=0.002594419926455438, P125=0.009254734844972098, P126=0.0118401529497117, P127=0.0027527452045429273, P128=0.0013320444796380473, P129=0.008745407764456163, P130=0.007843543967625094, P131=0.0012751628547864886, P132=0.0009229705608476067, P133=0.0016133509958021393, P134=0.0022059319182357695, P135=0.000964932101462825, P136=0.001142162399568904, P137=0.0032984070732314384, P138=0.0026161704877007337, P139=0.0010185657070279023, P140=0.0021364674512792965, P141=0.0014378073772981195, P142=0.000978414624327192, P143=0.0006038457400274675, P144=0.00036005787858850344, P145=0.00026311162673192234, P146=0.0004001559690193756, P147=0.000801026463971994, P148=0.0005025160207162776, P149=0.0004458603560440568, P150=0.0004212341456654464, P151=0.000448568016811254, P152=0.0004984609282212934, P153=0.000608553992102443, P154=0.0005807323180985213, P155=0.0004630836437176023, P156=0.00030793876529919647, P157=0.0003677977873637748, P158=0.0003464891501095513, P159=0.00032926751252224454, P160=0.00035564954067309807, P161=0.0005660477567075905, P162=0.0004922448132248178, P163=0.0002514126499773217, P164=0.00032265101193815595, P165=0.0002934800826351649, P166=0.0002654938586621153, P167=0.0001958238963197657, P168=0.00013235825467772984, P169=0.00020168097439195127, P170=0.0001974894097115407, P171=0.00018500694640402447, P172=0.00017023457674625382, P173=0.00020621542156155134, P174=0.00018242080085138863, P175=0.00019916626797411682, P176=0.00016242223336677706, P177=0.00014972407404715722, P178=7.803392327725363e-05, P179=1.0590168479343172e-05),Row( P0=-572.0850213064637, P1=24.36385252894647, P2=-35.221732000851055, P3=-5.597780296722453, P4=-24.265157758649625, P5=-15.418282177869285, P6=-9.633804581240598, P7=-9.588606603349538, P8=-4.296502144110883, P9=-7.897788683056242, P10=-0.19367956964799657, P11=1.306125909318669, P12=-3.0289266374573183, P13=3.582811877222654, P14=-3.518025516352264, P15=8.160391008876404, P16=2.190916809942893, P17=11.427190087814482, P18=9.97799887615059, P19=4.42939987773937, P20=5.438488508984215, P21=2.2195011680117336, P22=-1.512317308657751, P23=-0.11262095189569733, P24=-4.527961327175465, P25=-3.7324754914054368, P26=0.7416653640084684, P27=-1.2830237600090784, P28=-1.705453727754247, P29=-0.43631976062762173, P30=0.38757001068097535, P31=0.8504752973246031, P32=-0.13072615736111629, P33=0.6622597257160617, P34=-1.6039135210390985, P35=1.1916202230227375, P36=0.7816804730413595, P37=2.262875216939871, P38=-1.4518217472522064, P39=0.24963956822714964, P40=0.5169303034743057, P41=0.4734419000203704, P42=0.5123096070133176, P43=0.4805635008332561, P44=0.42586306023882087, P45=0.4009709911348986, P46=0.3995147760764892, P47=0.4497523307730444, P48=0.505158428676057, P49=0.511913986304843, P50=0.5738994348850693, P51=0.5567127215176936, P52=3.105431785527874e-06, P53=9.801073192024066e-06, P54=5.504560315663758e-06, P55=3.918121939886607e-06, P56=5.105279668261756e-06, P57=6.542940203913479e-06, P58=5.2849426607919396e-06, P59=3.5919973981144737e-06, P60=6.061027888410888e-06, P61=2.010117519018094e-05, P62=0.00017115055045095723, P63=0.0017392774214195564, P64=0.008207316283187641, P65=0.06894848775681164, P66=0.20172513745779208, P67=0.15805151911687887, P68=0.09249358103793728, P69=0.16912704436706694, P70=0.15295632207888202, P71=0.048039304207282744, P72=0.025143645022979326, P73=0.03223413434083662, P74=0.07363271995105243, P75=0.1403525012549481, P76=0.22676136598595442, P77=0.24852322804455776, P78=0.19091164258699067, P79=0.3035565819954474, P80=0.3893552202234775, P81=0.17007782413256828, P82=0.08964279537422368, P83=0.07521591529490004, P84=0.11053942273619798, P85=0.09342707054588516, P86=0.04265651715507265, P87=0.028388540832228814, P88=0.07055630846468769, P89=0.09187130674554447, P90=0.06279177912668116, P91=0.07072298658397777, P92=0.05188735537186302, P93=0.02362030025968963, P94=0.0550481440246433, P95=0.11884123311349254, P96=0.1525313115305907, P97=0.10787263159429002, P98=0.0741966188216325, P99=0.011848752504313678, P100=0.09842937307883434, P101=0.22927602677090092, P102=0.13211668731388063, P103=0.04857070840550481, P104=0.05259814003408868, P105=0.01659716534520595, P106=0.013058315091625453, P107=0.08765123148085606, P108=0.13906667396993114, P109=0.21968790416443748, P110=0.3676556860860998, P111=0.16833544270076153, P112=0.08253279627497287, P113=0.10492511042081931, P114=0.027878879271583563, P115=0.020932217205389528, P116=0.06354202594006672, P117=0.13859756584696617, P118=0.1754128394026354, P119=0.1441744786072248, P120=0.07246390685149186, P121=0.055124534308280085, P122=0.10545752772117688, P123=0.06232121570528359, P124=0.017426649615702595, P125=0.01734246001421109, P126=0.04071048720944465, P127=0.009206763311410869, P128=0.01321268890976114, P129=0.015378382065343827, P130=0.00494865298447908, P131=0.0034694491484191015, P132=0.003272883374117613, P133=0.0026375097538651315, P134=0.00456483933063875, P135=0.0037178555228550058, P136=0.00292018344732586, P137=0.007723919921190139, P138=0.006987658508845179, P139=0.013267007441533588, P140=0.016661937200971356, P141=0.019149177720311297, P142=0.009937983525308901, P143=0.006442660127161529, P144=0.0023599826305365354, P145=0.001531298835046965, P146=0.001654931384311862, P147=0.0021323798244476423, P148=0.002106281154924891, P149=0.003640676351676451, P150=0.00440986062448874, P151=0.0033211375898657552, P152=0.0039619369311007305, P153=0.003955181852392721, P154=0.00301395689416114, P155=0.0027157938883746045, P156=0.002514668560791622, P157=0.0006589969722757053, P158=0.00019211575320798594, P159=0.00030725398892424014, P160=0.00040349139086148404, P161=0.0003296671701616603, P162=0.0002542015404511787, P163=0.00023150238093904793, P164=0.00023595489934177634, P165=0.00024699430219563695, P166=0.0002407051695664665, P167=0.00026230705869777297, P168=0.00037477028469316554, P169=0.0004047769299402608, P170=0.0003739843315612517, P171=0.0005150153069644781, P172=0.00041762923748089767, P173=0.0003076132472927706, P174=0.00036228276371543037, P175=0.0001947269858510653, P176=0.00011491402704788268, P177=0.00013769748900814482, P178=0.00010541828615788303, P179=1.454020681264075e-05),Row( P0=-522.6352589146449, P1=21.621868151457925, P2=-41.82466927111333, P3=-1.6578294437302514, P4=-25.809532772484857, P5=-14.255305501296565, P6=-5.380394990941786, P7=-8.14615742291261, P8=-1.2937401568231084, P9=-5.095620321976025, P10=2.641781380831005, P11=2.1035730478101216, P12=-2.648506613050788, P13=2.0418418594901024, P14=-5.786242068506461, P15=6.369038965859697, P16=0.8827524837106941, P17=10.41898933301454, P18=8.945484021677279, P19=3.398134384924188, P20=4.989653980683177, P21=2.0243477966845203, P22=-1.6038379837604984, P23=-0.5732342879343744, P24=-3.749832476172841, P25=-2.6451588184907644, P26=-0.9019164831111949, P27=-2.727446463320994, P28=-2.1987828451442515, P29=-0.37788367849164173, P30=0.6704752070463182, P31=-0.34468058058565226, P32=-0.7544172290354738, P33=-0.30930450123826086, P34=-0.3095641933779305, P35=1.4414019232957271, P36=1.331031360374894, P37=1.6065708241123293, P38=-1.8516046011431078, P39=-0.1593799924931956, P40=0.623900902184338, P41=0.6082923322614252, P42=0.5854567607917855, P43=0.5297299857691782, P44=0.5334362379817321, P45=0.5590128205496749, P46=0.5758290004918892, P47=0.6098682801094328, P48=0.6561737588778113, P49=0.667091916545035, P50=0.6797952656403816, P51=0.6400974593922988, P52=5.550201531598448e-05, P53=4.030228913682844e-05, P54=4.145340185872039e-05, P55=4.70415229664058e-05, P56=4.3091464064172294e-05, P57=4.0488987808362684e-05, P58=3.122732417221355e-05, P59=2.572537603675671e-05, P60=1.936685641286177e-05, P61=1.854623941718519e-05, P62=0.00014844790253117912, P63=0.002799113971568992, P64=0.009078969606645062, P65=0.07505452790943856, P66=0.1981159869574657, P67=0.10779342342873663, P68=0.06111309800767151, P69=0.051629808129027115, P70=0.046331683067273276, P71=0.030995303070306842, P72=0.018474744022045256, P73=0.02112697425602564, P74=0.1267785842829468, P75=0.20675344863749126, P76=0.1739044833955618, P77=0.8481575758799612, P78=1.2295855163493559, P79=0.7036311720906312, P80=0.5874846225179817, P81=0.3759586251897137, P82=0.25907001051849615, P83=0.27829418544392776, P84=0.24580278344127565, P85=0.13587771837827414, P86=0.03520062410431245, P87=0.016173324368609444, P88=0.029021539989971066, P89=0.03961138797357631, P90=0.025293938809147862, P91=0.03197404366959336, P92=0.07301986633172086, P93=0.10708211998825516, P94=0.2145626792987371, P95=0.48127287328170126, P96=0.1351026547466832, P97=0.08011643959676923, P98=0.10476697178423271, P99=0.04103880158175305, P100=0.06379954882069198, P101=0.3203268349460296, P102=0.35259004088088636, P103=0.09127421206838497, P104=0.03997430873889184, P105=0.04580086956495347, P106=0.10455529267701028, P107=0.4750055729075283, P108=0.3601814237061711, P109=0.3524304351796904, P110=0.8948433711974659, P111=0.35605833612946286, P112=0.25841704717867864, P113=0.16354319655545185, P114=0.16151422246615768, P115=0.22858219105185143, P116=0.33964574599001873, P117=0.11826512087654212, P118=0.13039796610817855, P119=0.1521985568535451, P120=0.07824998074008312, P121=0.08564194160064802, P122=0.14462999105463847, P123=0.19465973114175572, P124=0.03922024753182693, P125=0.01876473011923099, P126=0.017317154715508768, P127=0.022503070453701143, P128=0.021536962006357498, P129=0.017222404229839408, P130=0.010806118985875432, P131=0.005814006150335247, P132=0.01154454540197628, P133=0.016221900208318863, P134=0.007191422660414516, P135=0.009874798884585739, P136=0.01286658401104457, P137=0.02247529578223707, P138=0.019213783774070785, P139=0.018163930856408477, P140=0.03761129956389161, P141=0.055160553155436845, P142=0.020972898392892004, P143=0.010551390000195993, P144=0.009719933362072148, P145=0.0032488193473661467, P146=0.004908249612829389, P147=0.00834858655168766, P148=0.005211797064997792, P149=0.011494768481894324, P150=0.01330699520801371, P151=0.011226184114721623, P152=0.01154057405387982, P153=0.008768557999521349, P154=0.012354184181159331, P155=0.005059240280039428, P156=0.001279127364586637, P157=0.0008210624561362189, P158=0.0005126646045655773, P159=0.0005707829451514407, P160=0.00030324411491703854, P161=0.0001667532628465857, P162=0.00010573344408310515, P163=8.061663414502242e-05, P164=0.00013024196533951887, P165=0.00015616130266048092, P166=0.00010811572993056722, P167=0.00011019686738707562, P168=7.935935178700145e-05, P169=7.797529016966913e-05, P170=0.00013480108241392352, P171=0.00019355844046293783, P172=0.00017278470848832271, P173=0.00015993702249816075, P174=0.0001034046406822074, P175=9.609967265385865e-05, P176=0.00013808367721847992, P177=0.00015498113488090695, P178=0.00011102533478089913, P179=1.3408800804030031e-05),Row( P0=-536.2401617485882, P1=2.5903443207194163, P2=-41.22792747693801, P3=-7.81685444651147, P4=-25.81776082167645, P5=-12.801177808480247, P6=-6.770312869280142, P7=-2.431365189053646, P8=10.962810448480665, P9=8.961857325399853, P10=12.042701845939602, P11=8.439253215178136, P12=-7.5914237703507, P13=-8.185535521784718, P14=-12.178526329685724, P15=-2.525235421762909, P16=-1.7897705053787194, P17=8.223346528472046, P18=1.6719284154316012, P19=-3.184329229443695, P20=1.3474333242216598, P21=4.8511539840177464, P22=-0.651612192089724, P23=-2.9389895706213043, P24=-5.65015151472816, P25=-2.6492154809668502, P26=2.214448396331406, P27=0.0829319555772821, P28=-3.744961591086649, P29=0.6464035918529928, P30=0.9172460455832073, P31=-0.4805269158689722, P32=-0.2315249020271514, P33=-0.5171942087874561, P34=-0.07315616100029096, P35=-3.0647175115729706, P36=-3.5152942185799616, P37=2.261138991682115, P38=0.29862578561502523, P39=0.8993624864294993, P40=0.6334289437567088, P41=0.5812338205689954, P42=0.5166814328544016, P43=0.4500207709300158, P44=0.46388607376162394, P45=0.44632884068904327, P46=0.4348551077481041, P47=0.4607385330161085, P48=0.5829062411495038, P49=0.7038419910325517, P50=0.6554730376833369, P51=0.602578155209596, P52=2.146487996463509e-06, P53=3.5278151543756045e-06, P54=1.0516705153242982e-05, P55=2.3992834909940863e-05, P56=3.6280995914933595e-05, P57=4.3517165096793736e-05, P58=1.9109113370507646e-05, P59=1.0432743676091477e-05, P60=1.187171930667469e-05, P61=1.2187965715238024e-05, P62=1.384460383128703e-05, P63=1.386663970772141e-05, P64=1.2958975558068209e-05, P65=1.783020446280167e-05, P66=1.8806202772541016e-05, P67=1.4395864200056094e-05, P68=1.2812068603391451e-05, P69=2.9929410996851655e-05, P70=3.864891772929165e-05, P71=7.008442251805055e-05, P72=0.00022147337937624987, P73=0.0008699344205156971, P74=0.011729934614583562, P75=0.06800147232828432, P76=0.2948642250448689, P77=1.2303726532757266, P78=2.1867285697427628, P79=1.3299570356867183, P80=0.5630480477129375, P81=0.4751685543041733, P82=0.6289507975551749, P83=0.6164790928455254, P84=0.2751412394265513, P85=0.06803015360311296, P86=0.007650301853608598, P87=0.0035678020252159644, P88=0.0021819946012250732, P89=0.0011061016733625573, P90=0.0006432455365682294, P91=0.0006257777795724587, P92=0.0005933003503073979, P93=0.0003149296754918608, P94=0.0005454225012752848, P95=0.001270900482211216, P96=0.0028871427264122902, P97=0.11340990622975629, P98=0.49654940534777753, P99=0.6158268136163819, P100=0.3522477232037045, P101=0.37632728430756296, P102=0.0529646700207273, P103=0.01796554404988392, P104=0.030965231343109283, P105=0.047941655000656704, P106=0.05551320184321633, P107=0.03699461229711729, P108=0.019546620410590004, P109=0.013648222124309922, P110=0.02480551854785466, P111=0.16098650383707422, P112=0.6908409621769797, P113=1.2849392911449027, P114=0.4617328280282812, P115=0.08222400663591964, P116=0.07024036446242561, P117=0.04727462017426411, P118=0.021601099047733087, P119=0.010893409961723902, P120=0.052720450203275794, P121=0.11530378373286218, P122=0.09613382889403416, P123=0.02041602595125431, P124=0.003646548113673204, P125=0.05227579952352336, P126=0.0525272569077382, P127=0.034977798326034856, P128=0.012942547779534758, P129=0.004910163080407539, P130=0.0033444515301943928, P131=0.005665972183791269, P132=0.007602853581110299, P133=0.0035118119873350324, P134=0.003979827130237546, P135=0.0022846438812665817, P136=0.0024042567633341515, P137=0.011517020360950899, P138=0.009063331879343457, P139=0.02958919676317538, P140=0.011326060058535035, P141=0.006969103780365048, P142=0.006847243015910268, P143=0.004099797781825524, P144=0.0015114509934271657, P145=0.0009675230778910204, P146=0.002156939326156304, P147=0.004171899954769912, P148=0.0016777408246736087, P149=0.0017552537758331343, P150=0.004181766219061994, P151=0.0035298145355108683, P152=0.002532825762845281, P153=0.003100738515854089, P154=0.0013090555067212467, P155=0.000853297869248702, P156=0.0006212405275182692, P157=0.00043544751017981565, P158=0.0004918906544562305, P159=0.00038609131888343856, P160=0.0003584008853792024, P161=0.0003962596104340412, P162=0.0002965844483199467, P163=0.000204824518766851, P164=0.00015634800204871873, P165=0.00014627751857411106, P166=7.116050594237745e-05, P167=5.60922979668851e-05, P168=7.43679444701976e-05, P169=0.00011275813661409004, P170=0.00014712602659762005, P171=0.0001370904459596932, P172=8.179640189870326e-05, P173=9.506987199976056e-05, P174=5.109456658547164e-05, P175=3.913257685238204e-05, P176=3.157573436924052e-05, P177=3.109122757353259e-05, P178=2.8277588280174537e-05, P179=6.606019453419035e-06),Row( P0=-685.2024268732713, P1=48.84474794531344, P2=-14.66985602923662, P3=1.187613744744093, P4=-14.860577406966225, P5=-4.082103235962119, P6=-6.7662173812783495, P7=-4.299385851971382, P8=-3.0673931041174916, P9=-7.327836910719354, P10=-4.319163990110235, P11=-2.4037832930294027, P12=-2.33583224790986, P13=0.36780615120394505, P14=4.87197142975028, P15=9.383529910605128, P16=5.248506257620126, P17=10.743357518084506, P18=4.702578528733258, P19=3.1909193067009105, P20=0.15213490051019243, P21=0.865328590934057, P22=-1.1667880202401728, P23=1.813054529707696, P24=-0.734083611015887, P25=-2.7281492883015375, P26=-3.7235821026769194, P27=-3.524318352418007, P28=-2.456013602216113, P29=0.2038279497282256, P30=-0.4490139794530998, P31=-1.2291441695714551, P32=-0.30000176474771445, P33=0.32376538495747786, P34=0.2839318319284076, P35=-1.9666519099951214, P36=-2.0011915620508915, P37=0.13816480189398192, P38=-0.8841685974099265, P39=-0.058254681579618724, P40=0.6887453829091987, P41=0.6471380582231353, P42=0.615306223757203, P43=0.5380994597485684, P44=0.5349653989435322, P45=0.5621474625867734, P46=0.5722445319692613, P47=0.6099371326383239, P48=0.5737807278796424, P49=0.5950987492911083, P50=0.631089559695443, P51=0.6416051335747308, P52=4.3206636039787095e-06, P53=1.4783016439987925e-05, P54=1.6136773085392224e-05, P55=1.3758138336689022e-05, P56=2.0585059697462282e-05, P57=3.725349109606543e-05, P58=1.610571824260913e-05, P59=4.019742212659678e-06, P60=4.064023456493817e-06, P61=1.2483162604968097e-05, P62=0.0011058936816456886, P63=0.004667987069635969, P64=0.005169818706574205, P65=0.015754685545383684, P66=0.03567123063184281, P67=0.0880059468649239, P68=0.058229332334968284, P69=0.02715264149111871, P70=0.008698253766510422, P71=0.0027112156770876055, P72=0.009239294989996746, P73=0.009865911734546172, P74=0.002255130156652968, P75=0.0011314904202794384, P76=0.0008689574731202138, P77=0.001810903409759406, P78=0.003176216976122332, P79=0.006432168574203032, P80=0.008932449321922307, P81=0.006907274338375526, P82=0.0054254252964482655, P83=0.009499423946979746, P84=0.009145460112034511, P85=0.016575336950311652, P86=0.03858564552306328, P87=0.014968933424396503, P88=0.0025385731280569645, P89=0.0008587299187136174, P90=0.0012032263001641916, P91=0.007226530012654798, P92=0.015968705470301037, P93=0.004175261866446833, P94=0.001586140683091803, P95=0.011293239402396576, P96=0.009649422689349877, P97=0.002708351669470556, P98=0.003140056584588027, P99=0.0009843942620466643, P100=0.004615269572489976, P101=0.003931369668562758, P102=0.0010952574732577595, P103=0.0006910537351691018, P104=0.008958691501805612, P105=0.00871696765908797, P106=0.006565048347176866, P107=0.007876412536109873, P108=0.0025544185589300426, P109=0.00027791788692497734, P110=0.0009133805013666152, P111=0.013468396490462215, P112=0.010466985212380205, P113=0.0038948611719947577, P114=0.0010447141357459412, P115=0.0001925329273006738, P116=0.002697004448534998, P117=0.010133875273163721, P118=0.005006224937516006, P119=0.0022960492176626593, P120=0.0004241384263445217, P121=0.0022218122493717386, P122=0.0025759642829170436, P123=0.0004782028950581976, P124=0.00019464194974496416, P125=0.0007915401350136658, P126=0.0021920787957399987, P127=0.0004885028788298226, P128=9.302320984530833e-05, P129=0.00014091367842179465, P130=8.636203794562486e-05, P131=8.56493611925834e-05, P132=6.26906882867528e-05, P133=4.286995040437965e-05, P134=3.044633694582365e-05, P135=0.00010029200444055655, P136=6.422770130695944e-05, P137=8.09914763355041e-05, P138=0.00015072911570191815, P139=0.00016494251039743312, P140=0.00017694409381504458, P141=0.00021081242750393338, P142=7.774145732346649e-05, P143=7.802067221816285e-05, P144=3.9961032012935596e-05, P145=5.145038764134457e-05, P146=2.8135434900017055e-05, P147=3.755436180954156e-05, P148=3.643058317448892e-05, P149=5.083396047091757e-05, P150=3.7176796276211564e-05, P151=4.0756313918276824e-05, P152=2.8862465396086712e-05, P153=2.4641199727922494e-05, P154=1.754422310815038e-05, P155=1.5018908953552275e-05, P156=1.0361749187878032e-05, P157=1.5544732043584454e-05, P158=1.798869659106188e-05, P159=1.3950246770254704e-05, P160=1.2295855064223236e-05, P161=1.1184641803163774e-05, P162=1.1545853658612385e-05, P163=1.4064894996349147e-05, P164=1.3919547012984356e-05, P165=2.4256074708289793e-05, P166=1.0577597093089458e-05, P167=1.545998994474898e-05, P168=3.814059389035116e-05, P169=3.7861974475779375e-05, P170=6.0949275719527934e-05, P171=2.8736680451833585e-05, P172=1.696790980674746e-05, P173=2.708007922469479e-05, P174=1.767193932688801e-05, P175=1.1839142016105444e-05, P176=4.4275006225719085e-06, P177=6.472392367933266e-06, P178=3.5417397986592513e-06, P179=5.973956899249755e-07),Row( P0=-669.7688607939333, P1=51.22970283257063, P2=-26.308307407033627, P3=-1.922682616813586, P4=-20.320190639781888, P5=-6.6342814376455665, P6=-13.269028577429253, P7=-3.2947371910706242, P8=-3.280356496817291, P9=-11.309758463403156, P10=-8.672008000961855, P11=-2.892592816832147, P12=-9.11509718877368, P13=-4.9502394845559445, P14=-5.823488042639703, P15=2.177611160749367, P16=-1.0525088672784835, P17=3.5295193720804354, P18=4.1162699268513245, P19=3.073814318479835, P20=2.2247408727578155, P21=2.7677995100418986, P22=-0.7174564242595968, P23=1.4226610827624486, P24=-1.9626792263213793, P25=-3.9194621301059573, P26=-3.7817984077998825, P27=-0.5755352096186435, P28=-2.784459867125132, P29=0.37328000037736614, P30=-1.114637912381668, P31=-0.9958427111989092, P32=-1.612381146741773, P33=-1.423096932633839, P34=0.1980724294473045, P35=-0.4485612131654427, P36=-1.0678784576232356, P37=1.8279108845670955, P38=-0.8221478568491185, P39=1.4612077524977023, P40=0.5669169434416673, P41=0.5384891183069461, P42=0.49889303120932044, P43=0.4552068044809522, P44=0.48932702242124054, P45=0.5004978687434023, P46=0.5521302494412291, P47=0.574344243410186, P48=0.6012391020339597, P49=0.6072290351422605, P50=0.5880173206589907, P51=0.5720800039479783, P52=1.1805449850393313e-06, P53=2.0054806040963416e-06, P54=1.9943856093883927e-06, P55=6.572623651519844e-06, P56=1.6519481390702127e-05, P57=3.091278274669755e-05, P58=4.9462707960217294e-05, P59=8.234094329534489e-05, P60=0.0001252672162462756, P61=0.00032139605918435094, P62=0.0024051200526001527, P63=0.008171015566307891, P64=0.015458184002989048, P65=0.034042776890515505, P66=0.03383131469227147, P67=0.025362856199228245, P68=0.010852413709437852, P69=0.006537884191988654, P70=0.0033420170324576707, P71=0.0008627445030040056, P72=0.0005569832078182334, P73=0.0006104348323119691, P74=0.0011440477607850236, P75=0.0035330250295001965, P76=0.008164137879810478, P77=0.011818719482684745, P78=0.01280270602442676, P79=0.012386653393132921, P80=0.01337841722633763, P81=0.014304736578177163, P82=0.03407663898811976, P83=0.03650159889869409, P84=0.026770580231952047, P85=0.023187119487988097, P86=0.015668507622524017, P87=0.011545785749602636, P88=0.008593321939213344, P89=0.0061947482063521376, P90=0.005294866580817825, P91=0.004803165130305972, P92=0.0033087439018151714, P93=0.001190557536891334, P94=0.0012897462208513362, P95=0.002570772208211256, P96=0.001822456888240323, P97=0.00270369191548284, P98=0.010219154678842782, P99=0.007793071374966105, P100=0.003558863600600697, P101=0.003377579357158513, P102=0.003713391678418913, P103=0.0024398529321581095, P104=0.005553523811110269, P105=0.003156478198622631, P106=0.0032846487891214525, P107=0.0021742142591597334, P108=0.0021993469135931634, P109=0.0011812284429454933, P110=0.0017171014962837113, P111=0.0032368599295751806, P112=0.0057911392760662565, P113=0.0071143675444063055, P114=0.0035407273994020124, P115=0.0035525222909609582, P116=0.003943282173330266, P117=0.0021687724476459353, P118=0.0008609490811904245, P119=0.000558359402392036, P120=0.0010835058801024816, P121=0.0023666563565752597, P122=0.0008152568361669107, P123=0.0010058532242134953, P124=0.0009777255070469455, P125=0.0006220473073543573, P126=0.0005809771282106601, P127=0.0005671684938262624, P128=0.00043581596694826727, P129=0.0003758824352270974, P130=0.0002543231681706867, P131=8.177208808523828e-05, P132=0.0002076178790079103, P133=8.364592101469577e-05, P134=5.893684708999665e-05, P135=0.00012892857300536353, P136=0.0001232186362990095, P137=0.0002960602296180175, P138=0.00022908934573056609, P139=0.00035906348631601406, P140=0.00025784480702612336, P141=0.0002568886691782688, P142=0.00013224628613106918, P143=0.00010030051931313023, P144=7.310476009264014e-05, P145=8.515092376851234e-05, P146=8.708434663153608e-05, P147=0.00012723086445636273, P148=0.00020888802613879554, P149=0.0001492830286691938, P150=0.00017234695439333528, P151=0.00015759307637854065, P152=0.00011605297961843892, P153=6.345444031222682e-05, P154=5.237516395423883e-05, P155=5.746076609301362e-05, P156=4.757744324000668e-05, P157=4.615705655071441e-05, P158=3.307845421382126e-05, P159=2.5785965773442292e-05, P160=2.2772148798183452e-05, P161=2.1262709272579556e-05, P162=1.805555859204527e-05, P163=2.2655975432444556e-05, P164=2.209467500068469e-05, P165=2.3612052578466216e-05, P166=1.7915258855203606e-05, P167=1.4033555563272996e-05, P168=6.416394264231422e-06, P169=7.5322985529141185e-06, P170=6.2800052196808286e-06, P171=5.750254411153074e-06, P172=5.087129966008088e-06, P173=4.25407944289342e-06, P174=3.6020599770147792e-06, P175=2.4771535461216335e-06, P176=1.7969168223321307e-06, P177=1.5642371493879912e-06, P178=5.181223236804592e-07, P179=5.974459268505347e-08),Row( P0=-603.9358973330131, P1=34.47773531596271, P2=-30.89681671157077, P3=-6.587879551687569, P4=-22.676896348710873, P5=-6.818089331121611, P6=-13.161765308343867, P7=-6.529069286804706, P8=-4.322237297317676, P9=-12.718353824700868, P10=-8.692645885578703, P11=-3.490696483790329, P12=-9.503296537129263, P13=-2.627711151159122, P14=-2.943499863127296, P15=4.163532220387103, P16=1.0175113107977405, P17=7.588655854432197, P18=3.933770893813313, P19=1.9758497938859556, P20=0.4431584022004819, P21=-0.11189158721614151, P22=-1.7854622783910927, P23=-0.15317985701728198, P24=-2.1187835223032816, P25=-2.918321807551415, P26=-3.7522604916715183, P27=-1.9358624080682862, P28=-3.6138559565590773, P29=-0.12465728577938216, P30=-1.0125840399004538, P31=-0.12660865321863843, P32=-0.7535204729574491, P33=-0.9905803945477931, P34=-0.4976861607009125, P35=-0.9157127298273087, P36=-0.7858025407325184, P37=1.899163645220798, P38=-1.1314650031098543, P39=-0.08680067299566975, P40=0.5512558991263842, P41=0.544331694581299, P42=0.5082372748680672, P43=0.4414840489424929, P44=0.4488510615406249, P45=0.47503761739190614, P46=0.4974421101591971, P47=0.5024545828592383, P48=0.5256258353799976, P49=0.5418679510020039, P50=0.571104804684377, P51=0.5412975260653669, P52=7.466149309622174e-07, P53=1.594995622661274e-06, P54=8.962714682583316e-06, P55=2.4222367259340814e-05, P56=4.282164198236376e-05, P57=3.542774444200539e-05, P58=3.2991335986374034e-05, P59=4.617545522978411e-05, P60=9.296627900728017e-05, P61=0.00029334906598204485, P62=0.0033151921255332602, P63=0.010483184805445921, P64=0.011776843646080604, P65=0.04179272194013874, P66=0.12682257397742938, P67=0.19965656358395165, P68=0.08331991592861045, P69=0.0478506452260493, P70=0.032598629902879234, P71=0.009106495719659368, P72=0.004433216561627081, P73=0.00573338108077302, P74=0.007490394569781825, P75=0.005128180248398365, P76=0.002263178419988985, P77=0.006896439471019683, P78=0.017416246595283312, P79=0.03841755902685447, P80=0.05390670163414299, P81=0.034729631347541506, P82=0.026202645253846555, P83=0.054697411544696976, P84=0.12636380702042796, P85=0.11809194840172209, P86=0.03642866039066449, P87=0.017933857909839365, P88=0.008845365722007751, P89=0.004397335661265471, P90=0.007048750445659669, P91=0.018216780231736146, P92=0.02060916761894027, P93=0.010487371037670952, P94=0.014979830640784512, P95=0.026973641263881794, P96=0.014948323572382741, P97=0.010839176373333124, P98=0.020280790354636282, P99=0.013380162639579756, P100=0.006795774176263364, P101=0.020532976173175857, P102=0.017555521126158484, P103=0.015404593362796183, P104=0.012126073890171106, P105=0.00545099144927923, P106=0.006466587239621524, P107=0.0071439748856178325, P108=0.010178954756541693, P109=0.00670976973284747, P110=0.0037169106379297363, P111=0.009132775629595791, P112=0.015191130730185035, P113=0.008781768589009168, P114=0.008909139593173757, P115=0.008298873193439141, P116=0.004995869621309479, P117=0.0037662941969080653, P118=0.00646478966321546, P119=0.015497296792179692, P120=0.006853878435692285, P121=0.004804447407308484, P122=0.006219050833398904, P123=0.004548661326422882, P124=0.004391288665110712, P125=0.001789429011611257, P126=0.0020083505529902433, P127=0.0016932071127326192, P128=0.0021186052371457713, P129=0.001182377230529601, P130=0.000707875834078436, P131=0.00019973221247919173, P132=0.00025397942474862493, P133=0.0004288112014396596, P134=0.0007714648084775132, P135=0.0010800343097203745, P136=0.000728994684530855, P137=0.0009941551576618999, P138=0.0007404678000114788, P139=0.0008376343489780484, P140=0.0009120406890103142, P141=0.0010149844769738532, P142=0.0008323322880297486, P143=0.001140775776854488, P144=0.0012169625939896088, P145=0.0003049176695117178, P146=0.0002890684311313566, P147=0.00025781385787225383, P148=0.0004117480863352874, P149=0.000527768932955069, P150=0.00041068963473126283, P151=0.00037672281161712194, P152=0.0003138488759745679, P153=0.0002840145329071902, P154=0.00017950884466363948, P155=0.00020879807128191608, P156=0.0001352776286304681, P157=0.00011995119754667072, P158=0.0001241197714897539, P159=0.00010161193435218621, P160=9.217942441149125e-05, P161=0.00011872596045754654, P162=0.0001193333022655176, P163=0.00014667614397337043, P164=0.00020723015725045794, P165=0.00025341419825286463, P166=0.0002148777924764861, P167=0.00011056885562159578, P168=6.243901125122007e-05, P169=6.980291655496815e-05, P170=4.79671593918402e-05, P171=7.677048567127109e-05, P172=4.124128457782963e-05, P173=3.165617762671322e-05, P174=2.2334968806401378e-05, P175=1.5850042212237458e-05, P176=1.0793338538637312e-05, P177=8.06299537442381e-06, P178=4.057014631777742e-06, P179=6.957425312677987e-07),Row( P0=-657.2278499273805, P1=44.03942466275879, P2=-24.75263262477403, P3=-2.4599620734728394, P4=-17.25329347399902, P5=-5.6913062986552445, P6=-11.66605143811842, P7=-0.6696914170686934, P8=-3.272415751580923, P9=-10.98264963036763, P10=-6.779404621254855, P11=-4.017381059181428, P12=-6.780181747111637, P13=-1.6450176554375637, P14=-2.7933129029765538, P15=3.413640215789905, P16=-1.1067781552788016, P17=6.019357996883738, P18=3.5332270003794983, P19=5.655979435227158, P20=3.55831322989455, P21=2.8075116912033344, P22=-1.1576494669276238, P23=0.7636543791539845, P24=-2.930603638359542, P25=-1.4956094847222017, P26=-1.1782882340735297, P27=0.9270260573436856, P28=-2.74361940843365, P29=0.10759984475467743, P30=-0.8033831770255365, P31=0.7892940713129554, P32=0.47283095797816166, P33=-1.2302526581435218, P34=-1.8070778163960006, P35=-0.9248380451541686, P36=0.8669421027237574, P37=1.3851483999025402, P38=-0.03216063916158782, P39=-0.3211002145047674, P40=0.5038239415588437, P41=0.5124014701079252, P42=0.5060926981388317, P43=0.45003159009176025, P44=0.4456469962720947, P45=0.4650596269689444, P46=0.4917023110265475, P47=0.5045227089487154, P48=0.5456363625756763, P49=0.5435888494499431, P50=0.555195230352191, P51=0.5034190256920615, P52=1.8398425213015271e-06, P53=7.138915300872928e-06, P54=9.005670047695868e-06, P55=6.238583948917561e-06, P56=1.3473751886439804e-05, P57=3.840358343507677e-05, P58=3.3747852788966804e-05, P59=5.327400206715859e-05, P60=0.00021867136485707623, P61=0.0006626554047181033, P62=0.002059627405104077, P63=0.004991209160410978, P64=0.010122691311497064, P65=0.03360208955114092, P66=0.0405480490369332, P67=0.013150667695541001, P68=0.006479900917856431, P69=0.011347424269507238, P70=0.01858010588524971, P71=0.006966412086673518, P72=0.0007362493334451863, P73=0.0005626428234866042, P74=0.0007771498906910605, P75=0.0012815171322994778, P76=0.0029640284547788995, P77=0.015556973706665306, P78=0.02452481177091611, P79=0.02123237765027522, P80=0.018207561493317186, P81=0.0081571333826853, P82=0.004438409883232458, P83=0.009909901958810556, P84=0.04182421690057671, P85=0.0384569313934161, P86=0.01327362398084517, P87=0.008846690080620449, P88=0.015054881200452239, P89=0.013037912706191132, P90=0.007738220339901341, P91=0.017811201732909825, P92=0.020317075938642973, P93=0.00724085200001114, P94=0.0037017736398535003, P95=0.00384197351997387, P96=0.0018498633366902063, P97=0.0009557193435016262, P98=0.0018414449077789603, P99=0.0014411118223753262, P100=0.005096911215349808, P101=0.012620494704423427, P102=0.004356456144618309, P103=0.0016330235215202132, P104=0.0017617309743545705, P105=0.0024291763600655957, P106=0.003788261001800452, P107=0.005311782694287024, P108=0.007075898497603333, P109=0.004288140319800916, P110=0.0019621368764949137, P111=0.0017791531713380081, P112=0.003421171948721113, P113=0.004931303634045621, P114=0.0034802625392634278, P115=0.0029011629894471363, P116=0.0021683188506019277, P117=0.0019543524529779263, P118=0.002142155470463085, P119=0.0026803885681252497, P120=0.000491485922025628, P121=0.00036214553322336306, P122=0.001842715585248944, P123=0.0015089633366029693, P124=0.00023527035213104794, P125=0.0005595568507956736, P126=0.00132205347745691, P127=0.00017811060103551082, P128=0.0002855038293471249, P129=0.0005585186849055517, P130=0.00022390419517448023, P131=0.00015328018494732107, P132=0.00016852840751226868, P133=4.637383647449957e-05, P134=0.00012546410422179595, P135=0.0003222496140651688, P136=5.662729200135697e-05, P137=0.00018759526887247418, P138=0.0001301923370932153, P139=0.00038559587944488543, P140=0.00021090473022671686, P141=0.00041537773027814766, P142=0.00010242290141530647, P143=0.0001252252754698538, P144=0.00015251511333838795, P145=5.3527724847120996e-05, P146=6.450449500561189e-05, P147=5.479300882468622e-05, P148=5.8211508501425424e-05, P149=8.959011568847479e-05, P150=0.00010289744997705242, P151=8.748031543320954e-05, P152=9.264236830395318e-05, P153=0.00010231224181799245, P154=8.699058896140834e-05, P155=8.493645043462249e-05, P156=9.086278453728132e-05, P157=7.366660896045045e-05, P158=6.659102461926943e-05, P159=4.641984093437788e-05, P160=4.0097910875485064e-05, P161=2.8327823330366424e-05, P162=2.8434599698783704e-05, P163=3.925375731871718e-05, P164=5.847038893041564e-05, P165=6.082734520143235e-05, P166=5.314808246567237e-05, P167=2.2312204504227293e-05, P168=1.4200910113758563e-05, P169=7.803186448207445e-06, P170=1.223951758230019e-05, P171=1.3310767126509864e-05, P172=1.4552868594733884e-05, P173=2.1381928672835756e-05, P174=1.3031180264553196e-05, P175=7.977815115259278e-06, P176=3.671832103380857e-06, P177=2.2121877071159645e-06, P178=7.988713891325048e-07, P179=1.38189029096319e-07),Row( P0=-578.1851340920116, P1=35.87099014920371, P2=-32.83311873585021, P3=-12.650403812427475, P4=-21.73896260614182, P5=-11.952749344299884, P6=-14.061397807958354, P7=-4.351417808901161, P8=-6.327537195625796, P9=-14.171284450980124, P10=-7.5596329985301844, P11=-5.166031699382296, P12=-11.07842060040688, P13=0.5243689042156325, P14=-3.5809418860340685, P15=8.689313301005852, P16=7.884494169378459, P17=10.539820372327538, P18=8.172617820443483, P19=2.054377096585183, P20=1.6419461414629346, P21=-0.12588285764145815, P22=-2.4864816867229873, P23=-1.1913746822651876, P24=-2.9834997449315765, P25=-6.765112709845056, P26=-2.7105661487748, P27=-3.0151627210424077, P28=-1.546060067553183, P29=2.4957191624933905, P30=-2.392147296120481, P31=1.255273546720832, P32=-0.9780238489767769, P33=0.6121374854613542, P34=-2.6005632447582676, P35=-2.724491911506213, P36=-2.5827508463650752, P37=0.543621866406328, P38=-0.24737355595017724, P39=-0.9352050974597178, P40=0.6540805575323737, P41=0.6494848931522088, P42=0.6430608405653098, P43=0.5433588540448706, P44=0.4892897419502834, P45=0.5117052264766775, P46=0.4963940941860438, P47=0.5253918943820087, P48=0.5734593633359836, P49=0.6015667126259827, P50=0.5942797181992839, P51=0.6162664869721642, P52=5.570096632487279e-07, P53=2.637651606912644e-06, P54=7.555580535343053e-06, P55=4.402678389707675e-06, P56=3.781815576519226e-06, P57=7.466101118975159e-06, P58=1.2582670929876839e-05, P59=1.4817141208381479e-05, P60=5.732816482498628e-05, P61=0.00022035067984590566, P62=0.0031669915464665795, P63=0.011250933646288397, P64=0.01246421164570708, P65=0.029299206836652894, P66=0.05708163205868345, P67=0.08961311134462262, P68=0.1690515892224097, P69=0.16988331658969744, P70=0.05774002934992011, P71=0.007054811663229054, P72=0.0017992959160299741, P73=0.0011166366386097502, P74=0.001211714978184652, P75=0.0016889143227535413, P76=0.002470343016128632, P77=0.01606267864960357, P78=0.025634189917050406, P79=0.030476608203499265, P80=0.0616769579666999, P81=0.07609381026279016, P82=0.08753907418714785, P83=0.25279998463872183, P84=0.432869211977202, P85=0.3885211666623754, P86=0.12833518849463119, P87=0.08441446095409079, P88=0.04264857096581985, P89=0.018889473250900464, P90=0.01634061099829684, P91=0.02410522250496453, P92=0.03163673557745355, P93=0.019773355623827326, P94=0.016623790842306137, P95=0.0364864001824729, P96=0.03241966868408293, P97=0.027002628496029933, P98=0.04345808672432265, P99=0.03677496003322933, P100=0.024861862852870443, P101=0.03135349957653119, P102=0.022537819244672696, P103=0.014365822783372584, P104=0.019080257622873676, P105=0.009159844174025749, P106=0.010061725654005545, P107=0.012764545641628083, P108=0.010622795476492766, P109=0.004655200436825695, P110=0.0024755469571058008, P111=0.0027933615500542125, P112=0.022423840623725023, P113=0.018891915855144627, P114=0.012074866046796319, P115=0.01674569596521532, P116=0.008988048478078721, P117=0.00438231575744206, P118=0.006176027044266453, P119=0.007305268498089115, P120=0.0038749403158522236, P121=0.0052097373820505265, P122=0.0027179854604104767, P123=0.002364718717655403, P124=0.0028512882072358526, P125=0.002834083492511201, P126=0.002384045193822991, P127=0.0011935836903436666, P128=0.0008150149102632702, P129=0.0028088260151063545, P130=0.0023470452811860366, P131=0.0016601702987349642, P132=0.002641807096720201, P133=0.0035295783023540648, P134=0.0019120434940142245, P135=0.00039361726720642937, P136=0.00038692699184337143, P137=0.0009597367374731561, P138=0.00042766181181741903, P139=0.001332147472330411, P140=0.001667405167992833, P141=0.001571102850957927, P142=0.0008577229732518445, P143=0.0003820003531338709, P144=0.0002143471063576828, P145=0.00019372279340152372, P146=0.00015500980822739628, P147=0.00023346999436692076, P148=0.0003770969262107764, P149=0.0005577495049818926, P150=0.0013686820512277568, P151=0.001309510133325771, P152=0.0015270928823952678, P153=0.0004531501876781031, P154=0.00015516053523472698, P155=0.0001012802273188058, P156=0.00013206088453329208, P157=0.00015225321120451135, P158=0.00019359038882069463, P159=0.0003222895884461475, P160=0.000473609653964146, P161=0.00020586838237046884, P162=0.00023681577658757393, P163=0.00035559476242925757, P164=0.0005441760275317902, P165=0.0007380821837846715, P166=0.0008048615671121786, P167=0.0007519788881282917, P168=0.0006109221119040355, P169=0.00029669454299187793, P170=0.00035251583049952817, P171=0.00041390966260520043, P172=0.0005278573397355223, P173=0.000570601196260239, P174=0.000343417472645902, P175=0.0002153395222700594, P176=0.0001889326515119525, P177=0.0001968770372148629, P178=7.983453908004094e-05, P179=4.95803112753541e-06),Row( P0=-654.3933967090076, P1=40.580284881009916, P2=-28.565634416745446, P3=-5.512147474540038, P4=-20.650878488451546, P5=-10.939698424374223, P6=-11.460929426937817, P7=-5.461333534121412, P8=-5.320170113898124, P9=-14.334628415457528, P10=-6.685357557075854, P11=-2.820935984321415, P12=-11.719608265435385, P13=-3.482184980091985, P14=-7.206618979460354, P15=3.27830647510806, P16=2.2518202505173557, P17=8.36984932999027, P18=7.42660654728878, P19=6.8659557216556015, P20=6.263696162140213, P21=2.143535750965803, P22=0.8250254047608583, P23=1.6087981291056799, P24=-0.6271120746941607, P25=-3.081014864812977, P26=-2.0920627287530555, P27=-3.8176873074196367, P28=-4.066927357892446, P29=1.793623757488648, P30=-1.541377299847512, P31=0.48179163577302353, P32=-2.0221308186204605, P33=-0.8713342732129462, P34=-0.09582923137167573, P35=1.04807964659181, P36=0.3265960516797691, P37=1.2108022185057428, P38=-1.200146480708918, P39=0.8970017632237071, P40=0.594257384804768, P41=0.5557077932860764, P42=0.5328134558913952, P43=0.4952029096456875, P44=0.47041812597473753, P45=0.4992536329380889, P46=0.503727219657058, P47=0.5747865696568012, P48=0.6291533694945814, P49=0.5952692106501858, P50=0.6189166042545649, P51=0.6224157984558862, P52=1.5830428436860764e-06, P53=2.0848146713440253e-06, P54=1.6161719173359821e-06, P55=4.0320008063035235e-06, P56=1.4628929832490458e-05, P57=5.6983351231907294e-05, P58=9.305805253797782e-05, P59=3.9216537868218794e-05, P60=2.8492556319739053e-05, P61=0.00015171963808105796, P62=0.003403629475286326, P63=0.013911267963471054, P64=0.014360188815621408, P65=0.04815016228031832, P66=0.09891311394682085, P67=0.0480231726821829, P68=0.0207201958992131, P69=0.010292158937691923, P70=0.0026967620102915585, P71=0.0005345191965091022, P72=0.0004186572442752311, P73=0.0004077564006243572, P74=0.0015755721523131438, P75=0.005149355739231813, P76=0.016251792008030785, P77=0.020555018352562137, P78=0.01065301544412393, P79=0.01676468428946475, P80=0.04852259021807141, P81=0.08446526814226467, P82=0.053280981980267844, P83=0.02278864623970185, P84=0.025625806286548404, P85=0.023310666824349242, P86=0.009953934324649817, P87=0.007943141643169768, P88=0.02142811124609732, P89=0.015374791323446982, P90=0.003392264940832717, P91=0.006451125156474176, P92=0.012403836014333215, P93=0.007127833900004137, P94=0.009603244551316144, P95=0.016143939115176947, P96=0.017995740342253198, P97=0.0187568284753719, P98=0.03633330159129232, P99=0.011021391717981294, P100=0.011445395982695885, P101=0.0178404790907725, P102=0.01122805745110667, P103=0.009052775902328964, P104=0.013749688244787173, P105=0.006156214678819659, P106=0.004441057364145585, P107=0.007984438999913544, P108=0.005661665627157061, P109=0.0017750771441986163, P110=0.0013592318228021138, P111=0.0009820578363470005, P112=0.0030671074961596957, P113=0.008201905504475034, P114=0.006597459163628485, P115=0.005633378319978018, P116=0.005038352701674822, P117=0.004381203439224443, P118=0.004920058922315862, P119=0.005225864513317805, P120=0.004667374679005002, P121=0.0031698253439725915, P122=0.0018362300347054203, P123=0.0008878656117569833, P124=0.001448266687480131, P125=0.001308804128091288, P126=0.0009049726698139335, P127=0.000606890859197207, P128=0.0011462826145087468, P129=0.0003928376159238963, P130=0.0005019510924908403, P131=0.00035907437434192456, P132=0.0002482608362536594, P133=0.00045011818500481285, P134=0.0003829448754366698, P135=0.00027330512389506294, P136=0.00018359820743188508, P137=0.00021466878525211336, P138=0.00025830240262115025, P139=0.0004078819755958634, P140=0.00032250757333356734, P141=0.00039619411501549235, P142=0.00022270645357584583, P143=0.00013302172864143136, P144=6.540132667024266e-05, P145=6.0865124238463797e-05, P146=4.997172632750754e-05, P147=7.82454540152935e-05, P148=0.00012921027676882207, P149=0.00021032407212667525, P150=0.00032183767193962363, P151=0.00020968331859809393, P152=0.00018198552230589223, P153=0.00010026801709127038, P154=4.4871159650453554e-05, P155=5.035555804695226e-05, P156=0.00010515950658523691, P157=0.00012142323621576881, P158=7.972748992918818e-05, P159=0.0001251851262458935, P160=0.00019363366419659285, P161=0.00010236094343486059, P162=6.953298761647459e-05, P163=0.00010432841721374995, P164=0.00016449642919026816, P165=0.0002489248460493582, P166=0.0004458943722620342, P167=0.0003894326874863944, P168=0.0002215169827848645, P169=8.671760770852885e-05, P170=9.974651285294501e-05, P171=0.00016126852337606067, P172=0.00023355296367306034, P173=0.00021736730192820978, P174=0.00013769244521571574, P175=8.193208063596415e-05, P176=0.00022119770117169285, P177=0.00018334087096339113, P178=3.770239792706804e-05, P179=2.712712126847908e-06),Row( P0=-599.1123719402332, P1=13.157495215586163, P2=-39.57288119157012, P3=-7.609032022947325, P4=-22.282395485707163, P5=-16.41100604000399, P6=-10.984731775008564, P7=-4.031414855107988, P8=-0.5498194917302605, P9=-5.2410405605600605, P10=10.029504371123995, P11=16.059848689449694, P12=10.848691613991086, P13=11.224375645976776, P14=2.3069613421364132, P15=-4.541191364249719, P16=-14.407016165645732, P17=-0.024611608205021437, P18=-0.663227529556494, P19=1.6448697107059138, P20=-2.811261646942523, P21=-4.035021700025267, P22=0.1352627505909782, P23=7.89108146710412, P24=5.725930545294905, P25=-0.7220020494282735, P26=-3.3193747522093164, P27=-2.6531493621007862, P28=-3.952857026270867, P29=0.4157371247358116, P30=-1.3089775176033067, P31=-1.9219719076407207, P32=1.895221104870216, P33=1.2332690659189516, P34=1.1125723233380784, P35=-0.866850980801237, P36=0.6296899101363934, P37=3.8403895648393416, P38=-1.888515271901446, P39=-1.2050855750172316, P40=0.608405192945186, P41=0.6393683605627901, P42=0.5699358990439408, P43=0.47191071420171166, P44=0.5002589934384585, P45=0.6260905439415332, P46=0.6630040510262079, P47=0.6235337280142694, P48=0.5717259336775828, P49=0.5653860747234721, P50=0.5703233627199382, P51=0.5376737276217699, P52=3.054031250176719e-06, P53=4.606421006685361e-06, P54=4.173212090110912e-06, P55=5.4868504181516946e-06, P56=6.271588838172704e-06, P57=5.231762661235631e-06, P58=5.10233419582606e-06, P59=2.994591190367124e-06, P60=3.052104989732231e-06, P61=3.0633674201859324e-06, P62=3.263700366992299e-06, P63=3.17977773751683e-06, P64=3.880392194894006e-06, P65=9.244730805210611e-06, P66=1.9320971987973423e-05, P67=3.0427246128999284e-05, P68=0.00024267479693601326, P69=0.0027004946815071557, P70=0.007900398754839047, P71=0.0138118760056666, P72=0.05616805378589799, P73=0.11525626261276826, P74=0.3021690933131591, P75=0.47752914525408036, P76=0.28076630659237317, P77=0.19664302038678141, P78=0.11114383935075739, P79=0.0462770342124761, P80=0.012171997438350391, P81=0.003354985067105734, P82=0.0013138322554665791, P83=0.0006037199267736771, P84=0.0005306593121243107, P85=0.00032359823224677577, P86=0.00012045192259909993, P87=0.00013852670911352618, P88=0.0001991204965863612, P89=0.000624819262747483, P90=0.004873703228011103, P91=0.06655654748686773, P92=0.2290535627090749, P93=0.13898442577601874, P94=0.10512281207789305, P95=0.0714102586389541, P96=0.04686133546632788, P97=0.04921988669953084, P98=0.02437386382413431, P99=0.0026840110773752816, P100=0.0006841238214466918, P101=0.00047558206144888006, P102=0.00019918488751834756, P103=0.0008388565004279019, P104=0.008241621720170074, P105=0.11702433851113632, P106=0.10006065696692304, P107=0.08257892246859551, P108=0.08235943092282341, P109=0.054633950452518766, P110=0.022610386446842382, P111=0.0011927017167731107, P112=0.0018680325505758394, P113=0.019383187177130643, P114=0.042822387289327954, P115=0.016117715315214057, P116=0.005112443376183572, P117=0.007145608679952741, P118=0.033715755535770865, P119=0.0315285699989073, P120=0.013979979372298999, P121=0.020113086394181304, P122=0.024979583643946694, P123=0.01100597638182068, P124=0.003546542667158522, P125=0.002183512668102589, P126=0.0030153947788311673, P127=0.0039028440619515103, P128=0.0027245281609378394, P129=0.002384358148292304, P130=0.0018157496938825712, P131=0.0024023985273348776, P132=0.0015005383111367048, P133=0.0005180563729353427, P134=0.002549729062590327, P135=0.0014445526444274096, P136=0.0007496530753473873, P137=0.0006752932528930564, P138=0.0029916716167523766, P139=0.0029807100826288544, P140=0.0014390469669931917, P141=0.0037391440564112184, P142=0.0025357947328296443, P143=0.0006260630259625452, P144=0.00027862449327001586, P145=0.000325223049455143, P146=0.00023339544888247853, P147=0.0003405118479684539, P148=0.00040197903559928796, P149=0.0004359189214190135, P150=0.0005275532352869069, P151=0.0004227671776163071, P152=0.0004532887025549032, P153=0.0010389298846925103, P154=0.0009654982033980466, P155=0.0007756024297445247, P156=0.0006084861430810852, P157=0.0005735780977016185, P158=0.0009409791467644531, P159=0.0009962700810193022, P160=0.0012522134100242054, P161=0.0013063250198200363, P162=0.0008455625354676109, P163=0.0004946102201405878, P164=0.0005541750950498785, P165=0.0005750018994239421, P166=0.00042625688212484465, P167=0.00024700928563117406, P168=0.00020409394086839922, P169=0.00022045021120455368, P170=0.0003306536622399236, P171=0.0005822232480277156, P172=0.0005033657338750054, P173=0.0003615211293119872, P174=0.0001638221268118299, P175=0.000269731843310197, P176=0.0004975308651785992, P177=0.00046558505649035094, P178=0.00025785712855234445, P179=5.415677482259594e-05),Row( P0=-626.9729267475656, P1=40.027076110083655, P2=-26.60934006746079, P3=-0.7479176034947477, P4=-20.136561921633504, P5=-10.283307863500514, P6=-8.873768826603708, P7=-5.196264378556779, P8=-5.126665409366036, P9=-11.636438220340906, P10=-3.7898487305785786, P11=-0.9441766685408165, P12=-4.546525786304014, P13=0.7058581372164074, P14=-2.7478919461413898, P15=5.049316178626387, P16=-0.9482359731595008, P17=6.230787925282819, P18=2.243105641817233, P19=0.4237807195121836, P20=0.005820287885475979, P21=-0.9919799970871149, P22=-1.4999291433163657, P23=0.7762901304255287, P24=-5.1225580722377915, P25=-4.913089000079217, P26=-2.911746862515041, P27=-1.7489232264455064, P28=-1.1741266900860328, P29=-1.1991223196625418, P30=-1.6903628794748575, P31=-0.5419668958607615, P32=1.2006695990916667, P33=0.3587879839086409, P34=-0.570297561378278, P35=-0.9204917120428224, P36=-2.3683886787224404, P37=1.0640312553517661, P38=0.5951543655917686, P39=1.2810452480324277, P40=0.6836758458174135, P41=0.6647484727342451, P42=0.643907740950168, P43=0.5988730242854743, P44=0.6179122492733756, P45=0.5854679218920155, P46=0.5805859666116266, P47=0.6584159739332183, P48=0.6862982093903663, P49=0.6934534892040385, P50=0.7060114708554296, P51=0.6519044907891212, P52=1.2830091871502884e-06, P53=4.783545566610482e-06, P54=9.954854851532021e-06, P55=1.7299636845063333e-05, P56=3.455352813122565e-05, P57=7.112286929431957e-05, P58=6.903333431193687e-05, P59=0.00010246319158954395, P60=0.00016770210912642585, P61=0.00019115772523920646, P62=0.0006809722720118778, P63=0.0036636542216057894, P64=0.008160217241635933, P65=0.0252145931343021, P66=0.0345891908840881, P67=0.0658536708898336, P68=0.07828801933408255, P69=0.032004442260204796, P70=0.023075792445111484, P71=0.017908763039007555, P72=0.012453757217341899, P73=0.004517542582125714, P74=0.007429058884717433, P75=0.021076065541069753, P76=0.028956575931130924, P77=0.021866348644591423, P78=0.01458683717552771, P79=0.020902760297819397, P80=0.022365960744764223, P81=0.012272590973697504, P82=0.014342455200463183, P83=0.05347499035298375, P84=0.11909306463862745, P85=0.08636492537082174, P86=0.033922590405069356, P87=0.03572717506813153, P88=0.03220610402504356, P89=0.017837934631458302, P90=0.019006308650349996, P91=0.03507189554587709, P92=0.029642368264163495, P93=0.005869817823730873, P94=0.01569549188205659, P95=0.01667991682468221, P96=0.016830849022357697, P97=0.03348148233104495, P98=0.033214079544412974, P99=0.019125036182490725, P100=0.048812467393602114, P101=0.05689784391951341, P102=0.02561917361400712, P103=0.005635847849967282, P104=0.003018371211975892, P105=0.08133487225155606, P106=0.09973444423617826, P107=0.06703574748038878, P108=0.033766697181813854, P109=0.006707299860809346, P110=0.002716748679020924, P111=0.0014664951936067836, P112=0.036547245221273894, P113=0.025626844890873204, P114=0.0049751010424584595, P115=0.005747772355775701, P116=0.006853765333238543, P117=0.0406528046985085, P118=0.1267965885366427, P119=0.027641755564250275, P120=0.0062522073417978444, P121=0.012617126444887217, P122=0.019055426280644314, P123=0.008335555660603304, P124=0.004597179205888522, P125=0.012574976021318282, P126=0.00891235631526034, P127=0.0013219034522869253, P128=0.0011029999949569516, P129=0.002984328321373149, P130=0.0011306190466358467, P131=0.0009991500686074987, P132=0.0015911934630996592, P133=0.0004311567787464902, P134=0.0007411634507663142, P135=0.0003754526776395317, P136=0.0004873627398787591, P137=0.0013042117240557082, P138=0.0010599196497174596, P139=0.004037576378028373, P140=0.0015152882389452287, P141=0.002953517638888858, P142=0.002753694984285113, P143=0.0008192169303394159, P144=0.0007664754788290336, P145=0.0002454705652161028, P146=0.0004209036944886943, P147=0.00030260076546997954, P148=0.0007333302760371876, P149=0.00044606834284193283, P150=0.0008863407472112365, P151=0.0005572538329879124, P152=0.00046491453190283246, P153=0.00046926173697800527, P154=0.0003089359394211831, P155=0.0002625348100127173, P156=0.00017294053925031512, P157=9.759955280952934e-05, P158=4.892173356857443e-05, P159=4.4980026395422164e-05, P160=7.927816644857412e-05, P161=0.00011233965022537274, P162=0.0001238376716973693, P163=7.654832921606592e-05, P164=8.597457959436819e-05, P165=0.00011851494837066161, P166=6.00665840920312e-05, P167=4.817696987515676e-05, P168=6.947883025570647e-05, P169=7.787128986152873e-05, P170=6.16100398108198e-05, P171=8.993971692471666e-05, P172=6.271851485999343e-05, P173=4.944393489905326e-05, P174=2.831882427717959e-05, P175=2.8086682343167352e-05, P176=2.7151239670838942e-05, P177=1.5307422701043243e-05, P178=6.208099237169197e-06, P179=7.502127779583102e-07),Row( P0=-633.9964084074439, P1=32.30564940195314, P2=-32.84316107233058, P3=-2.761387424453057, P4=-21.308152750467137, P5=-12.263554712853105, P6=-11.703337344905707, P7=-6.856299046152227, P8=-4.6749112740532945, P9=-11.753623046524599, P10=-3.662175506743709, P11=0.0443217201830122, P12=-2.8398722189969403, P13=2.9969116343847526, P14=-1.6630632780013874, P15=6.456965255932148, P16=-1.2178482691306673, P17=4.6280549087487755, P18=3.3859688626183293, P19=2.5718626948305072, P20=4.450676460520017, P21=2.843234812591564, P22=0.9074116060399229, P23=2.4600675024529317, P24=-3.8534392092521923, P25=-4.426842545756132, P26=-1.6869941320522785, P27=-0.6475352503813657, P28=0.273924174271066, P29=-0.9867911871315188, P30=-2.1582330539324817, P31=-1.9373943514846421, P32=-0.04014477568107861, P33=0.7218190533064851, P34=-1.5982171734770032, P35=-1.6631888257965988, P36=-3.01680439703939, P37=1.8309680093195606, P38=0.5655898834566101, P39=0.4013057684383657, P40=0.5023502930198116, P41=0.5420787122356312, P42=0.524820270889255, P43=0.5005266322181634, P44=0.5086335971239908, P45=0.46744828668272975, P46=0.4959353666455697, P47=0.5697198932058339, P48=0.5824692054081412, P49=0.5978417015625792, P50=0.5568177756605668, P51=0.48812143818602455, P52=1.1246624089100111e-06, P53=3.5592077345120145e-06, P54=5.773434929991922e-06, P55=5.4518415718835055e-06, P56=5.430481376837003e-06, P57=8.880246167609476e-06, P58=8.910457622406372e-06, P59=7.313353668208869e-06, P60=7.308558154747134e-06, P61=1.601457923181361e-05, P62=0.0009257097012523631, P63=0.005625679895878139, P64=0.012233652656354914, P65=0.02583160118700219, P66=0.024355579323176824, P67=0.03786515883952778, P68=0.08976464675058424, P69=0.0709223840844981, P70=0.04677187387137247, P71=0.032730116334130754, P72=0.017919905726697796, P73=0.009551947150784572, P74=0.017131904904322266, P75=0.02282155730423066, P76=0.010912851047198953, P77=0.013890421929714061, P78=0.026059031990326494, P79=0.039513712255740456, P80=0.03671767214929973, P81=0.0133808729228935, P82=0.005971547306275954, P83=0.022608634562525286, P84=0.09170992887195405, P85=0.08283718470082493, P86=0.04770916530893859, P87=0.05357559612500308, P88=0.05238941111057891, P89=0.028212091012229525, P90=0.020645689512863382, P91=0.038242366180239455, P92=0.02349109274092407, P93=0.0054066766410542965, P94=0.007566445657999417, P95=0.020938609907858844, P96=0.018164585903225443, P97=0.02546332048303767, P98=0.03350338464607823, P99=0.021886417154222234, P100=0.04559356841566658, P101=0.05733672670534175, P102=0.028103350196664944, P103=0.006331380884722316, P104=0.00459716050801904, P105=0.0448219608436281, P106=0.10027841221390411, P107=0.06395852921373112, P108=0.03911102710100182, P109=0.01748263145075403, P110=0.006724330835641284, P111=0.006842589179670214, P112=0.028210309899727896, P113=0.026084111668116115, P114=0.010315885757626422, P115=0.006893701847017132, P116=0.027247010952864775, P117=0.04241990752403625, P118=0.10065882184885429, P119=0.03370629752466354, P120=0.01834519071625333, P121=0.03266727781309531, P122=0.008028567945824678, P123=0.004989000969227636, P124=0.007785643582599298, P125=0.008190137797969885, P126=0.0026603873264891007, P127=0.00168532294661544, P128=0.0019183492486700097, P129=0.001985187093343934, P130=0.0008975378696114698, P131=0.0015608069116121643, P132=0.002019334216134732, P133=0.0008877738348565573, P134=0.0007697958499183925, P135=0.0005475292049193323, P136=0.001243817606115772, P137=0.0014803806137015357, P138=0.0012654695454579526, P139=0.002860453450718183, P140=0.002597240461327736, P141=0.0027995739846360273, P142=0.0013319367868607965, P143=0.0005688044544058248, P144=0.000517527772498283, P145=0.00027313964959320533, P146=0.0003618977678459771, P147=0.0003707423644989435, P148=0.0007182255680207442, P149=0.0006033279318462184, P150=0.0009357301601619927, P151=0.0005839695419452944, P152=0.0005187757341089943, P153=0.0005249512435468474, P154=0.0004242976918333569, P155=0.00020719448563255067, P156=0.0001296634736084321, P157=4.754756556066814e-05, P158=4.5136137730935065e-05, P159=6.627282727697676e-05, P160=9.782474102704053e-05, P161=0.00017391880276995745, P162=0.00018121293827041399, P163=0.00012941697124378796, P164=0.0001524316098369438, P165=0.00014319410365343525, P166=0.0001154717845946025, P167=9.142937938247713e-05, P168=0.00010241374827325158, P169=6.531519332186205e-05, P170=8.378046545673762e-05, P171=8.395477648078461e-05, P172=6.054648936796886e-05, P173=5.143897810572925e-05, P174=4.5776243922053485e-05, P175=3.966492819125977e-05, P176=2.500195736769985e-05, P177=1.7081188419286488e-05, P178=1.061421301598206e-05, P179=1.2254103627838575e-06),Row( P0=-680.20021810844, P1=40.31368592692923, P2=-20.757980859657117, P3=-2.4146722335685924, P4=-16.56840063747823, P5=-5.774226236106484, P6=-10.279858931065124, P7=-7.150527132474075, P8=-5.992053715802458, P9=-10.698369066491894, P10=-8.294968237504921, P11=-6.10238371189197, P12=-5.125024142870617, P13=-5.979769648719034, P14=0.6072426371011427, P15=3.767342311240398, P16=-0.5195908392036069, P17=6.151911243336522, P18=4.165336924296681, P19=6.43970605919072, P20=4.6370682547069695, P21=4.9295380593729385, P22=0.6098452130518893, P23=-3.341870181223249, P24=-6.68676973860114, P25=-6.656390074506285, P26=-1.8447701317440979, P27=0.00883679905532563, P28=-1.3018661175792057, P29=-1.5660224284128426, P30=-2.5578285872532467, P31=-1.7389891496712788, P32=-1.8794242978406446, P33=-1.9800705343380038, P34=-2.471329326763794, P35=-1.3303650346075158, P36=-2.363322778642272, P37=0.3963154320271205, P38=-0.08739500522310079, P39=2.806304292533503, P40=0.4731424573252726, P41=0.46913933559526994, P42=0.4813160185126383, P43=0.5095102721390112, P44=0.4444621212274027, P45=0.37749228452485295, P46=0.39763160697685007, P47=0.46276816550382066, P48=0.541144897823187, P49=0.6015685914761075, P50=0.5612518700246124, P51=0.4966586598323341, P52=7.224564759942091e-07, P53=1.507528798286761e-06, P54=5.028154733806028e-06, P55=1.5418861177287908e-05, P56=1.966646267477008e-05, P57=1.5318191863978326e-05, P58=1.0718519368300242e-05, P59=7.920304551008909e-06, P60=1.039710140570812e-05, P61=2.3733881290992146e-05, P62=0.00037287288161935006, P63=0.0049733593574491025, P64=0.0266282855300629, P65=0.03767900068254671, P66=0.02184353503019867, P67=0.013691591486442585, P68=0.009323715590928093, P69=0.029459726341544035, P70=0.05111679449621746, P71=0.016520968698679413, P72=0.0004676070955467486, P73=0.00011608715117112443, P74=7.367454906226514e-05, P75=0.00022829641977571805, P76=0.0015181498328068579, P77=0.005922474868350264, P78=0.014081341985204134, P79=0.010308049461707429, P80=0.004383128337259498, P81=0.0018463398756258572, P82=0.0024988837410517177, P83=0.0016496657061766146, P84=0.0010066123901771576, P85=0.0006844496832609157, P86=0.0008270027842789233, P87=0.007951787673318831, P88=0.04008225023291077, P89=0.01817928131807077, P90=0.003996526067802084, P91=0.0015791686806728093, P92=0.00044053940391219096, P93=0.0003409726620828829, P94=0.006999008684236323, P95=0.00833489044950278, P96=0.002015261632467663, P97=0.0012352971492488123, P98=0.0012083104061028076, P99=0.002066153794965931, P100=0.004207044170187065, P101=0.005298806977880524, P102=0.0013017273637372481, P103=0.0035246086503562236, P104=0.006239101838353608, P105=0.004045282870926351, P106=0.00446699094565688, P107=0.0012149636928273449, P108=0.00034261067888862227, P109=0.0016537142426422042, P110=0.02737652263815075, P111=0.014249336823633222, P112=0.004103338706869567, P113=0.0008189729235898295, P114=0.00017506788367922145, P115=0.004979746451938641, P116=0.017226037532991758, P117=0.0034553465990964536, P118=0.001105683364929605, P119=0.0003699655906514925, P120=0.0013467786433366828, P121=0.0015543201735889929, P122=0.0004302330660263695, P123=0.00024097628932294473, P124=0.0005520639860011399, P125=0.00026293312766415965, P126=0.00021260917270542282, P127=0.0001829926045877526, P128=0.0001493406871777672, P129=3.390885832254048e-05, P130=5.0327290916682353e-05, P131=5.122290561144829e-05, P132=3.1696490631204795e-05, P133=2.419783819610438e-05, P134=0.00020223806572552275, P135=0.00010908662518444857, P136=2.612584216110818e-05, P137=0.0001387128788812824, P138=7.603105929303385e-05, P139=0.00011864163070783896, P140=0.0002520249378153101, P141=0.00016524319975297648, P142=0.00018528039423045304, P143=0.00017930288785151022, P144=0.00011002912801951608, P145=5.0786583456059174e-05, P146=4.99233450140232e-05, P147=5.23656372182141e-05, P148=5.130481549301659e-05, P149=3.171651203349815e-05, P150=3.0715937146228505e-05, P151=2.57000619846754e-05, P152=2.6523243469701387e-05, P153=2.2540177753132472e-05, P154=2.9500432098751767e-05, P155=2.6904399733684686e-05, P156=1.927347659620697e-05, P157=2.141102979608914e-05, P158=2.4295120223644977e-05, P159=2.2037313280016833e-05, P160=3.000510713187789e-05, P161=1.876505643047366e-05, P162=2.8042291983473194e-05, P163=3.447921991209397e-05, P164=2.1252562788440266e-05, P165=1.9565428733171215e-05, P166=1.337986191104327e-05, P167=1.0385969805345627e-05, P168=2.8073046987937423e-05, P169=3.575375974532287e-05, P170=2.8364513278616395e-05, P171=2.3982571348289828e-05, P172=1.7317558655681032e-05, P173=3.2875966859945086e-05, P174=1.8275825288136957e-05, P175=6.497072430667108e-06, P176=5.230130808769741e-06, P177=5.790830911259883e-06, P178=6.259109201376292e-06, P179=1.230240814193128e-06),Row( P0=-673.9247935251357, P1=39.776054667766736, P2=-26.35435268465729, P3=-7.746303061114926, P4=-16.687701419891088, P5=-11.791946007859549, P6=-12.569103653089421, P7=-8.766420373259654, P8=-7.821644600838448, P9=-12.25154840713209, P10=-8.5745804199192, P11=-4.819010141436769, P12=-6.3625481822939, P13=-2.2898272712345427, P14=0.14962079000739115, P15=8.734856738556333, P16=4.1360144182201, P17=12.467178342607038, P18=6.993642126588512, P19=5.939088411847172, P20=4.157885507853918, P21=3.281120730215376, P22=0.03679368768220636, P23=0.8377511331952365, P24=-2.480107650040723, P25=-3.617642494080009, P26=-2.413307229339013, P27=-3.6676612483047593, P28=-2.0728550621487427, P29=0.3530249277356512, P30=0.5968907123822428, P31=-1.4643371425776068, P32=-1.4225676409989692, P33=-2.143686273090198, P34=0.5181584341595822, P35=0.4391672606377643, P36=-1.1586045198324315, P37=-0.15959617415909974, P38=-1.725200006358809, P39=0.631851704699203, P40=0.6073465191914657, P41=0.562113292490543, P42=0.5529319867920223, P43=0.528343915766214, P44=0.4811249372036611, P45=0.462803706262652, P46=0.4878128716136342, P47=0.5340953395874634, P48=0.5773983619214729, P49=0.6162976455619612, P50=0.6214836139784957, P51=0.6307538182672039, P52=7.7694414221052e-07, P53=1.7343941182001076e-06, P54=2.1456053218608616e-06, P55=2.8805992034990254e-06, P56=2.750626546273959e-06, P57=2.911774607031157e-06, P58=1.8939718683463715e-06, P59=1.390574681877287e-06, P60=2.050148405253644e-06, P61=1.3696561039315141e-05, P62=0.0009956803230951347, P63=0.006163443845181085, P64=0.010744445531873282, P65=0.025489690758379223, P66=0.08312835629731201, P67=0.11603166887431732, P68=0.03665252409384792, P69=0.0164654864331601, P70=0.012406741618093074, P71=0.0065831930246962435, P72=0.0005548169597199728, P73=0.00015956853669469846, P74=0.0004289188752840206, P75=0.0016359485040416384, P76=0.0020109945934144646, P77=0.005293290612474477, P78=0.014505966078643146, P79=0.014293966272048026, P80=0.007215447702674325, P81=0.007004808088118694, P82=0.025304945349857545, P83=0.03703097972230988, P84=0.03144120787531864, P85=0.01831897597742684, P86=0.007004594047614717, P87=0.004165104486121158, P88=0.0032917493884955214, P89=0.003998373422494141, P90=0.005636272177217242, P91=0.004225884603591217, P92=0.0014190835195312965, P93=0.0033308340201099965, P94=0.007101378265950904, P95=0.012691410573555368, P96=0.00748426772454924, P97=0.007449574730336587, P98=0.012613999441394361, P99=0.007234362194686783, P100=0.002603855330017472, P101=0.003582107515967947, P102=0.006946988322371724, P103=0.0030050467935111517, P104=0.005331388552401328, P105=0.005736190995044961, P106=0.007420984562418018, P107=0.0019062739741155879, P108=0.003328856617723243, P109=0.005920900332234191, P110=0.006446955748310732, P111=0.00509017418201406, P112=0.003922469375145573, P113=0.001372646874499992, P114=0.001557028243830991, P115=0.00255446614758563, P116=0.0038037900863127482, P117=0.0023586872238945513, P118=0.0022001894940375175, P119=0.003067627466679868, P120=0.002478469926354348, P121=0.002274747650788574, P122=0.0006131347024572791, P123=0.0006553267334616504, P124=0.0015681075565729904, P125=0.0007246894521004386, P126=0.0001762461427052998, P127=0.0003066632428538455, P128=0.0007276251762425324, P129=0.00010704652186495128, P130=0.00011183804852182224, P131=0.00010427348010275637, P132=9.723103618364681e-05, P133=7.413777968666827e-05, P134=7.083037122291388e-05, P135=0.00016729089257205498, P136=4.3543973498679685e-05, P137=9.851989412165948e-05, P138=0.00012167555517158019, P139=9.32271296631804e-05, P140=0.0002499108162907796, P141=0.000193642763643777, P142=0.0003039242947312694, P143=9.796547637463108e-05, P144=3.151568333208941e-05, P145=1.6476359324136464e-05, P146=1.914523933240007e-05, P147=2.0752617571926433e-05, P148=2.8550162005325617e-05, P149=3.9401708332512725e-05, P150=3.3008635046712725e-05, P151=2.2016521080994103e-05, P152=3.0157346625595254e-05, P153=5.882508711059008e-05, P154=3.0403602671600816e-05, P155=2.0309124912092383e-05, P156=1.9613086739415726e-05, P157=1.899105465230448e-05, P158=1.991620986739329e-05, P159=3.386118701511806e-05, P160=5.697917312908803e-05, P161=8.567899317601165e-05, P162=0.00011064903916574071, P163=7.911134962805095e-05, P164=6.314093726399871e-05, P165=3.8019937048013595e-05, P166=3.6864259507444096e-05, P167=3.272267660187329e-05, P168=3.8943387179693095e-05, P169=4.813997658623863e-05, P170=5.730754576366698e-05, P171=8.351315146980663e-05, P172=6.959174597965419e-05, P173=6.932424660258938e-05, P174=5.4521494675024916e-05, P175=1.804425156057972e-05, P176=3.033590310416394e-05, P177=4.649061509809541e-05, P178=3.080607189874638e-05, P179=6.971845459071998e-06),Row( P0=-607.1620125845292, P1=20.70199607945774, P2=-37.6573952500629, P3=-4.668367658346261, P4=-22.36103479283413, P5=-11.879980927510543, P6=-11.218159758892861, P7=-7.381506991184077, P8=0.9494197522558038, P9=-5.367274582434145, P10=4.539380074637334, P11=9.379862358777572, P12=9.333857716382303, P13=9.54751474630388, P14=4.058810214375736, P15=0.4662256403109844, P16=-11.115996827285255, P17=-1.015684700046016, P18=-1.429158733383048, P19=-0.19526774522214527, P20=-1.735392165974215, P21=-1.43008509133557, P22=-2.0690327720248787, P23=3.1175906291269433, P24=1.125511408780259, P25=-0.6789159662885168, P26=-2.180374256369924, P27=-0.5308945228304552, P28=-2.730998315529668, P29=-0.022265400803755973, P30=-0.8360293528632744, P31=-2.614776573042851, P32=-1.157623491136337, P33=-1.4281916802836043, P34=-2.2229316011957914, P35=-2.246974024965096, P36=-0.7955620133028444, P37=3.9576496044322957, P38=0.20615893831676962, P39=-0.0020023563074890636, P40=0.5290381033250702, P41=0.5283213013289956, P42=0.4880181048726411, P43=0.45896539037873235, P44=0.5222898059381395, P45=0.5676855371536574, P46=0.5264438505529636, P47=0.4943249918159431, P48=0.5191356483438548, P49=0.5458427650519864, P50=0.49563969595420576, P51=0.45706834946644914, P52=1.3741673367645537e-05, P53=5.543415728854608e-06, P54=1.5475544214293394e-05, P55=2.418478026723511e-05, P56=2.045853680637292e-05, P57=1.4341540414125385e-05, P58=7.306295444549828e-06, P59=4.485232910560687e-06, P60=5.510089822744504e-06, P61=1.0130823247233477e-05, P62=1.355830640707689e-05, P63=1.0893939534439271e-05, P64=1.0123704890396613e-05, P65=2.3461981609461918e-05, P66=4.1582946002029895e-05, P67=6.342085355135804e-05, P68=0.0008621445704315503, P69=0.008078495723407785, P70=0.01534743628019222, P71=0.022388857802523886, P72=0.060523070592079974, P73=0.10081303977565888, P74=0.17612872413403213, P75=0.20392465817202998, P76=0.19565872199616866, P77=0.27481797707225464, P78=0.2093278096774985, P79=0.05738879542934415, P80=0.006969886006302509, P81=0.0009865906419480575, P82=0.00047629805885455865, P83=0.0003732067437793391, P84=0.0003782044735205287, P85=0.00028299736204058203, P86=0.0010654621478074673, P87=0.009419333494275746, P88=0.022814724696049993, P89=0.0294036969431147, P90=0.04097651126951984, P91=0.08680505256103697, P92=0.17010481175095254, P93=0.15000052162849725, P94=0.0760978061690439, P95=0.0538629962583597, P96=0.018624545563307258, P97=0.009540359007665321, P98=0.009023960394645307, P99=0.004022534338303228, P100=0.012581059600344764, P101=0.02803326069331189, P102=0.011819687674519512, P103=0.0047161192510570505, P104=0.008108581166558241, P105=0.1544766804740119, P106=0.12010874503987812, P107=0.046943483540104844, P108=0.02216681396699759, P109=0.005061715253070806, P110=0.006628330429277763, P111=0.025885158796863164, P112=0.02753007376918592, P113=0.015572698240940734, P114=0.10815957298175564, P115=0.053321016151999544, P116=0.01735688101672184, P117=0.021582470035921748, P118=0.007876000103630865, P119=0.0012813742090904922, P120=0.0033499151450778826, P121=0.01710670906343087, P122=0.017038330120140388, P123=0.008429570356353043, P124=0.0009064452959817387, P125=0.0015127926276943923, P126=0.0025161768412363473, P127=0.004353882852933277, P128=0.0009555619772375466, P129=0.0004139348331101874, P130=0.0006283624735109583, P131=0.002356720186505878, P132=0.0008798224381509743, P133=0.0002296495677233303, P134=0.00022588432409180524, P135=0.0009172064645048474, P136=0.004320387377282713, P137=0.0015079291742215077, P138=0.0010960479305335098, P139=0.002477548827498931, P140=0.003469116855701488, P141=0.0011045218395420615, P142=0.0013252392354207226, P143=0.001013768485054813, P144=0.0004137727794056942, P145=0.00044166577314565947, P146=0.00027353163246407935, P147=0.0003652652728890738, P148=0.000567283942812819, P149=0.0004286246328544134, P150=0.000465508988427835, P151=0.00039929282725721795, P152=0.0005341975248172461, P153=0.0006458315842762807, P154=0.0005229286709166858, P155=0.0005072616628346, P156=0.00015143490256358514, P157=0.0001576771725020651, P158=0.00012997656673022673, P159=0.00015087280385362066, P160=0.00014523241036542238, P161=0.00020235540210261275, P162=0.00013306716330987032, P163=8.731116401896783e-05, P164=7.51353547597858e-05, P165=6.573655294305759e-05, P166=3.771664574816628e-05, P167=5.6185676656829565e-05, P168=5.888019491372265e-05, P169=4.584561351492323e-05, P170=3.8054735870159375e-05, P171=4.4929764348151845e-05, P172=5.0534331535671516e-05, P173=6.089258084814465e-05, P174=2.979917727077555e-05, P175=2.266377605785976e-05, P176=2.2724534566301382e-05, P177=3.163875570031248e-05, P178=2.090376325661724e-05, P179=2.8823900708946513e-06),Row( P0=-533.3539646896885, P1=0.492481438454942, P2=-38.51748147316243, P3=-10.324947397834679, P4=-21.489382239639706, P5=-13.72764563692834, P6=-8.423414769050211, P7=-4.014149930633715, P8=6.757822448233594, P9=7.533679218253214, P10=12.309497123936648, P11=8.376117792986092, P12=-5.157827626484606, P13=-6.880444972862906, P14=-11.41254277183156, P15=-1.9587957036791397, P16=-3.499062078337122, P17=6.473174128155897, P18=2.264634959124593, P19=-0.04936735567896287, P20=0.6700720668642849, P21=3.3910922735038533, P22=-0.0047173276566771834, P23=0.07773492914309717, P24=-4.034368668628894, P25=-2.983651472653098, P26=-0.09365460351887368, P27=-1.1119818228141043, P28=-3.3026270676174376, P29=1.1048398511411557, P30=1.2556131625361353, P31=-0.32492451991207305, P32=-0.24857515793092275, P33=-0.8540958652260218, P34=-0.25411107232200625, P35=0.48354299661778516, P36=0.49815068354138414, P37=0.2681959824251666, P38=-3.6543914409179727, P39=0.7814531297804047, P40=0.588565395406465, P41=0.5676745989662668, P42=0.5345322345658244, P43=0.4373041892463924, P44=0.40528774253409255, P45=0.4166767608660131, P46=0.44158559101077843, P47=0.510996076619731, P48=0.5816434936709081, P49=0.6007514838229807, P50=0.6297944069590707, P51=0.5998143090054214, P52=3.2348372645335024e-06, P53=5.10072137697601e-06, P54=8.475950685912005e-06, P55=1.2517146939480242e-05, P56=2.148506461652749e-05, P57=3.264852235256614e-05, P58=1.8999219386004764e-05, P59=1.2599200321025881e-05, P60=1.3849708737121153e-05, P61=2.0217162325107057e-05, P62=2.843011610226198e-05, P63=3.091245771273918e-05, P64=1.8804815228878464e-05, P65=2.5907410979000107e-05, P66=4.4114431490702324e-05, P67=5.340859326660667e-05, P68=4.0409437230775584e-05, P69=0.0002639453627980465, P70=0.0010493984734553637, P71=0.0035549281057014126, P72=0.00624121321045792, P73=0.012059970017843777, P74=0.06663524451146897, P75=0.25449185309596745, P76=0.5210968909220759, P77=0.7698963715448977, P78=0.8290820914343094, P79=1.0348178199082025, P80=1.7814238301513865, P81=1.574619043763513, P82=0.8384101873841713, P83=0.1952660286503965, P84=0.09006092333018821, P85=0.05505378327168309, P86=0.16862675154173212, P87=0.9086395505045923, P88=0.8942875206378792, P89=0.06976489125802396, P90=0.003955746290851185, P91=0.0015753566214330005, P92=0.001056288364554745, P93=0.000941079105522977, P94=0.007612153140467749, P95=0.08257066786598834, P96=0.24275333482805087, P97=0.4404663676171619, P98=0.32169186031579083, P99=0.08618609957682369, P100=0.12758473925110891, P101=0.21095451807156015, P102=0.1578517324400952, P103=0.25759431339870104, P104=0.2986142068123365, P105=0.04734825685256594, P106=0.0041930044349390546, P107=0.007901137240117793, P108=0.14731449134169888, P109=0.24031446084223362, P110=0.23242342649478467, P111=0.10906656996429653, P112=0.15332288885189674, P113=0.14234968692927966, P114=0.11534781523792531, P115=0.12951965476227256, P116=0.07276570471269811, P117=0.06526674354255024, P118=0.18682005940200583, P119=0.3435772536984672, P120=0.12322218087384944, P121=0.1380086511077006, P122=0.09734944251701008, P123=0.04933716227936941, P124=0.01985740284461431, P125=0.09480318542416616, P126=0.14551691342112827, P127=0.007067466114599772, P128=0.0026972600907126357, P129=0.012874384879195398, P130=0.028582504516044247, P131=0.018630675004137263, P132=0.0075985516150759765, P133=0.002573196314895588, P134=0.00482547933333068, P135=0.02200466751475041, P136=0.04285349065990021, P137=0.038111055444631274, P138=0.00719612080175003, P139=0.018090472865761764, P140=0.0117150709216565, P141=0.01283140591181823, P142=0.03897067704286733, P143=0.00826806088384731, P144=0.0031376338091024685, P145=0.0009978109655671059, P146=0.0016734404290720077, P147=0.0029221736149146337, P148=0.0027185277568980326, P149=0.004378178938464326, P150=0.008228766078112225, P151=0.004975230502798327, P152=0.002143634105464752, P153=0.0026489507227539623, P154=0.010832125617818636, P155=0.005563799247852041, P156=0.0008671864099198027, P157=0.0008747118869973461, P158=0.001241588285629192, P159=0.0014093014912046006, P160=0.0014909993775759221, P161=0.0016458615181106, P162=0.0011577417089135493, P163=0.0008478239748881342, P164=0.0008615551985898295, P165=0.001038728029857452, P166=0.0005929020651378669, P167=0.00033732851595995414, P168=0.0003826031284344411, P169=0.0005647414839923959, P170=0.000631259651459586, P171=0.0007150455164708104, P172=0.001243419836265101, P173=0.0011933635454320403, P174=0.0007295214581157492, P175=0.0008129904950081998, P176=0.0007485800073927305, P177=0.000487576617668132, P178=0.0005524437761769405, P179=9.596680650154183e-05),Row( P0=-680.5926055860599, P1=41.23725246164702, P2=-19.274452140386554, P3=-7.151263304801944, P4=-15.063900923865047, P5=-8.95123012407186, P6=-13.21434638231035, P7=-6.933048467981443, P8=-7.963569093248984, P9=-11.473396740080336, P10=-9.15278637005714, P11=-3.5231598936501993, P12=-7.847391314625772, P13=-6.076879786653671, P14=0.592440937391204, P15=7.189055851093892, P16=2.971958309527307, P17=11.75179275054971, P18=5.985865585382732, P19=7.44731953247137, P20=3.1587254455028306, P21=3.311924116774018, P22=-1.072209240373441, P23=-1.557541590902069, P24=-4.2072815928698075, P25=-5.430574433866088, P26=-1.723603521109899, P27=-2.298337831919364, P28=-0.011792608782794986, P29=0.8719982175795951, P30=-1.5220753762599373, P31=-0.5951931708003119, P32=-0.21198652646785007, P33=0.33114383524709934, P34=-1.2410135291994409, P35=-2.5888198483541713, P36=-2.9114164989174323, P37=2.4880016382892496, P38=3.165218454359787, P39=5.53356822252035, P40=0.527681713258583, P41=0.45626277645746666, P42=0.42067944347895675, P43=0.4139385212299674, P44=0.3797726999835416, P45=0.3493306434422612, P46=0.40238177568560074, P47=0.491642515850408, P48=0.6401559713696446, P49=0.596829594881138, P50=0.5107954492796596, P51=0.502880380804521, P52=2.034740862884653e-06, P53=1.926558499240509e-06, P54=2.1716030270610395e-06, P55=3.0117115000032222e-06, P56=6.167012002599412e-06, P57=7.065790491582551e-05, P58=8.894423868987134e-05, P59=1.5963765569055574e-05, P60=3.280847437524333e-06, P61=1.5784655557943325e-05, P62=0.0002788170457675281, P63=0.0050905473759740435, P64=0.034556220779904834, P65=0.04514224168090068, P66=0.03927426682889529, P67=0.04327134908986761, P68=0.029695097091876503, P69=0.02836165246063863, P70=0.014776278367965669, P71=0.00234946083807926, P72=0.0003816140029643776, P73=0.00010775488358253079, P74=7.627038984494772e-05, P75=0.00011585681580530932, P76=0.0005669730892535207, P77=0.009238123202679287, P78=0.022038006530014004, P79=0.01499807764615723, P80=0.009000738039858347, P81=0.0061773522389889585, P82=0.007685274384588499, P83=0.014883908348010514, P84=0.022570038357809307, P85=0.017884536667712277, P86=0.007059724909255609, P87=0.0043355241568459445, P88=0.002378626255154106, P89=0.002546982869353336, P90=0.0039197956470701165, P91=0.0026049293554287816, P92=0.0008074276212579711, P93=0.0005686708661139953, P94=0.007151006499125404, P95=0.012028320272923568, P96=0.0062257467360761366, P97=0.009386082814897716, P98=0.008935170788160687, P99=0.004439032398499737, P100=0.0034061820316611474, P101=0.0036735867195667286, P102=0.002034661204569806, P103=0.002157326053778493, P104=0.004181888752932258, P105=0.010839425208996799, P106=0.009521545581516126, P107=0.0005567880639769961, P108=0.00010257907035619832, P109=0.00043170900999573824, P110=0.004296353043909119, P111=0.003215662939676464, P112=0.0023070370841200304, P113=0.0009321951885830193, P114=0.00017338281682574187, P115=0.0012869625531994387, P116=0.002592695859951159, P117=0.001604404672902525, P118=0.0020828097905419593, P119=0.0012105191565871774, P120=0.0012237173571489441, P121=0.0012923930077159227, P122=0.0005841565227325318, P123=0.0006607036894109644, P124=0.0005544809004776694, P125=0.0001983884411164448, P126=0.00026506361503594704, P127=0.00017048721087388802, P128=0.00012640190037752595, P129=5.361657862814226e-05, P130=8.136739319937163e-05, P131=4.8275052756922896e-05, P132=5.012667942146491e-05, P133=6.250319322627891e-05, P134=4.091997945725775e-05, P135=3.281685950963697e-05, P136=3.73970617175125e-05, P137=4.428172436452798e-05, P138=3.519574714356432e-05, P139=6.44067714728461e-05, P140=7.973074297665257e-05, P141=8.094930063563953e-05, P142=0.00012005729801051567, P143=7.216263897877215e-05, P144=4.236049723619503e-05, P145=2.1321774961425352e-05, P146=1.8799231061781827e-05, P147=1.92275230151694e-05, P148=2.3381301058945194e-05, P149=2.220976615799041e-05, P150=1.812736380932911e-05, P151=1.626875525506846e-05, P152=2.4178704101160715e-05, P153=3.6640436071674675e-05, P154=5.507947741180733e-05, P155=3.484770580722573e-05, P156=2.5188446165139695e-05, P157=2.9773994983841145e-05, P158=4.2802450544835436e-05, P159=8.345672665244821e-05, P160=0.00011704821327958997, P161=0.00020241362676436044, P162=0.00023566033696028558, P163=0.00016483664439994732, P164=0.0001016585327410742, P165=0.0001108016916200492, P166=6.950505729619675e-05, P167=3.659622526275844e-05, P168=7.854604427237518e-05, P169=7.698198389725408e-05, P170=9.103614447616591e-05, P171=7.250164951355304e-05, P172=3.1604937783672336e-05, P173=4.093438986171255e-05, P174=4.051782945246613e-05, P175=2.7324328957783005e-05, P176=3.1851365959018e-05, P177=1.7520148401015602e-05, P178=1.1170386670318194e-05, P179=1.3857262892291555e-06),Row( P0=-526.8237425714875, P1=18.41800260991443, P2=-40.32667226286493, P3=-1.667645966986998, P4=-27.174360160836567, P5=-13.520345077635891, P6=-7.982840358753782, P7=-11.471825731125652, P8=-2.6814686090465805, P9=-8.670839605382065, P10=2.1078231895606168, P11=0.087411143463099, P12=-1.4737193704301894, P13=2.7952480765656658, P14=-3.856882799462535, P15=7.390587967227458, P16=0.6295295353047485, P17=8.93196634901669, P18=6.229730463172094, P19=0.4386001146725976, P20=0.7868601946646607, P21=1.2091942718789528, P22=-1.0031885087368728, P23=0.5580311698219221, P24=-4.1942121513237325, P25=-2.908689542099789, P26=-1.5371708617392934, P27=-3.4815543284819697, P28=-1.5505822356537124, P29=0.20590461696997447, P30=1.8583330482825038, P31=0.1368935486208674, P32=-0.5277471787843391, P33=1.1290608074280961, P34=0.6489236859879464, P35=0.24886805812330995, P36=-1.233878512172688, P37=1.0888744812730178, P38=-0.2195161751844882, P39=0.6455170986618944, P40=0.5745055299796041, P41=0.545450020917943, P42=0.5173602835847058, P43=0.47854406075494166, P44=0.4689704434693903, P45=0.4585387556966447, P46=0.5059446585778058, P47=0.5069580744039459, P48=0.5439016757658501, P49=0.5976761519038328, P50=0.5267893680361853, P51=0.5283317670693746, P52=1.597788570497126e-05, P53=1.4983742161978097e-05, P54=9.547971189961838e-06, P55=8.72923624516526e-06, P56=1.5251999200894947e-05, P57=2.3276310559570453e-05, P58=1.3540158258637124e-05, P59=7.681631049169096e-06, P60=1.6145373342143532e-05, P61=2.7317264057137483e-05, P62=9.326558362678273e-05, P63=0.0014446595405988008, P64=0.009937846682339265, P65=0.03377046830403098, P66=0.11015087572304542, P67=0.22346424943225873, P68=0.19736638951377153, P69=0.12640268746111433, P70=0.06963492661637277, P71=0.04428615393592493, P72=0.11758550363786109, P73=0.12089516134909595, P74=0.14109011551491385, P75=0.17127423075423215, P76=0.14749072208026953, P77=0.49094229178919735, P78=1.112971194317168, P79=0.989895615214692, P80=0.32089494705602617, P81=0.15099267464923885, P82=0.11190809043928662, P83=0.13283800464637535, P84=0.20068261557008035, P85=0.15201171058614568, P86=0.056784610758605136, P87=0.04199453616278551, P88=0.04030597920621568, P89=0.02770318040333849, P90=0.030887825407449672, P91=0.04787480387202482, P92=0.06415348657437858, P93=0.05915296846691829, P94=0.16886244986413645, P95=0.40356120318152583, P96=0.20391856245539408, P97=0.04657429429415559, P98=0.051576191153233905, P99=0.04263712181947973, P100=0.10988527065801679, P101=0.1100543159347466, P102=0.044872034040421484, P103=0.08984001341809225, P104=0.15743893541089254, P105=0.0844098078756935, P106=0.06970459688679495, P107=0.12309720551697419, P108=0.1720073516928592, P109=1.1272852545552623, P110=1.2407742845171967, P111=0.30283533701929066, P112=0.35227949867685715, P113=0.39292063564910495, P114=0.17237569942509917, P115=0.06854926499771655, P116=0.16551552935263308, P117=0.06487320213665987, P118=0.056803941980629255, P119=0.1075853772545303, P120=0.1279877437424527, P121=0.17369433694584474, P122=0.12727716458769678, P123=0.06345942179001642, P124=0.028101303534278355, P125=0.044185795036989065, P126=0.019920845211887167, P127=0.04076140894025709, P128=0.03910444685284851, P129=0.01219976710658044, P130=0.008936848780683996, P131=0.00663784968839803, P132=0.01640940164830062, P133=0.01961414497487799, P134=0.01149309094509794, P135=0.007533990134995029, P136=0.006857842299803722, P137=0.021553590047780773, P138=0.015301326853778913, P139=0.010197608684712076, P140=0.01981161682885737, P141=0.03612141447865367, P142=0.02162552994329921, P143=0.0063340396573027275, P144=0.0033372468463008483, P145=0.0027101521934633716, P146=0.005184564944114927, P147=0.0036514888375683494, P148=0.007633243195387419, P149=0.01019300749475386, P150=0.01003926984222149, P151=0.011318535362191268, P152=0.011623857749014092, P153=0.008712658148218918, P154=0.005706226165294174, P155=0.002958147558771247, P156=0.000614992239988479, P157=0.0003931921891572515, P158=0.00041554888517880305, P159=0.00036829257412335686, P160=0.00024584312265696074, P161=0.00015373838543341137, P162=0.00013920087924217591, P163=0.00011012274391568051, P164=9.63031811105603e-05, P165=8.053108845838042e-05, P166=8.557019070336336e-05, P167=4.295811467879974e-05, P168=4.636895284743834e-05, P169=5.43290241486856e-05, P170=6.37644441224096e-05, P171=8.366968833940516e-05, P172=6.809943540155795e-05, P173=6.652003690400952e-05, P174=4.480275163290931e-05, P175=3.4844723313186785e-05, P176=4.59216489023483e-05, P177=6.195305264011355e-05, P178=4.064121500157895e-05, P179=6.079128870808426e-06),Row( P0=-642.4466222353026, P1=44.860056075914606, P2=-23.137354294388444, P3=-6.143328695779248, P4=-16.7582952594195, P5=-10.640009559935674, P6=-11.676053184167639, P7=-4.983278770281757, P8=-4.293689498965616, P9=-11.739087549604674, P10=-8.274703527625125, P11=-3.8108501168431794, P12=-12.012043167622856, P13=-5.468115785998369, P14=-8.236418096196276, P15=1.250420127749268, P16=0.38853966485360697, P17=7.494344709488903, P18=7.2586968647477335, P19=4.260838567549437, P20=3.7584587727856444, P21=0.404425149273206, P22=-0.43916384183901613, P23=2.628767019891739, P24=0.3268713444349794, P25=-2.7963872501881806, P26=-3.690484856297352, P27=-5.021806349032388, P28=-5.7630644888469345, P29=1.3684137976152608, P30=-0.253274399149162, P31=0.40589759453919066, P32=-2.4198395799957004, P33=-1.8007291529810197, P34=0.2921195786730034, P35=1.7271989871528413, P36=-0.09550653606375495, P37=-0.774305300160686, P38=-3.485494510429149, P39=-0.4808194766790353, P40=0.5459524321542643, P41=0.4881390826427701, P42=0.4831022079457574, P43=0.4164202344023073, P44=0.4082631667495004, P45=0.44748218713707316, P46=0.5131016228680306, P47=0.5598391446064898, P48=0.5421466426056762, P49=0.5116307408531575, P50=0.5488119932167062, P51=0.5781199322990908, P52=1.016229077545513e-06, P53=1.6031266679629033e-06, P54=2.9490334917073137e-06, P55=1.3507795081191985e-05, P56=3.858982658266608e-05, P57=7.765005721728517e-05, P58=5.984925071279329e-05, P59=6.1014302698418105e-05, P60=5.342132431382121e-05, P61=0.00027316214706277713, P62=0.004694576846803107, P63=0.015411750384191363, P64=0.01090275764338309, P65=0.034233348228700805, P66=0.13792711707135505, P67=0.1420475959164967, P68=0.0368389002218177, P69=0.005028639478612, P70=0.0012462765408283603, P71=0.0003237503891646363, P72=0.000251179133304235, P73=0.0003658687962670107, P74=0.003012822990357165, P75=0.009081619183004671, P76=0.00945991055600187, P77=0.010248759845790741, P78=0.00909384652347105, P79=0.011153665501887592, P80=0.03187993726940072, P81=0.06776307113727671, P82=0.09906211445842544, P83=0.0518557258512357, P84=0.0442030932677874, P85=0.025582384389347724, P86=0.0075321669787371094, P87=0.008978498417791848, P88=0.0049264183449589955, P89=0.002177476301515278, P90=0.0016706596745306125, P91=0.0034449652936379385, P92=0.005608393817267129, P93=0.007681438486939144, P94=0.014405097929030759, P95=0.025858050489663737, P96=0.01604085139329033, P97=0.018967555810114688, P98=0.015945832564224657, P99=0.003661044111082471, P100=0.004577811599760267, P101=0.02042500088734957, P102=0.01970071475633067, P103=0.006740273373804012, P104=0.012210553294123207, P105=0.005866973553293821, P106=0.004505642262085332, P107=0.00438604347228522, P108=0.014786293841185441, P109=0.012175228960825018, P110=0.005895650440832245, P111=0.0038354350287976937, P112=0.003597515292949401, P113=0.007476101577349706, P114=0.01298312638956513, P115=0.00817158886774962, P116=0.006633605545274657, P117=0.00543225388965533, P118=0.0053080492836753935, P119=0.00908269393046799, P120=0.002918905089907513, P121=0.002215253655158198, P122=0.0015857064158779919, P123=0.0015955798243199647, P124=0.001059837856792054, P125=0.0005914027104008786, P126=0.0011867825655976357, P127=0.0005581111368532206, P128=0.00037843412045322474, P129=0.00024797710165319435, P130=0.0005565754704485555, P131=0.0005909797180951302, P132=0.00021130026281318082, P133=0.00016650560735358068, P134=0.00022808804548595687, P135=0.00022622074253216286, P136=0.00013947636494127358, P137=0.00038555667664239945, P138=0.00029522243093614114, P139=0.00047543926598708704, P140=0.0004210308816325241, P141=0.0005630718088477465, P142=0.0004189920151358394, P143=0.00011230914505644587, P144=7.485976503629975e-05, P145=5.981383898230163e-05, P146=6.189589706549426e-05, P147=8.204776906691956e-05, P148=0.00012975678059729725, P149=0.00017320342708389188, P150=0.00017507230649378597, P151=0.00017177999795251717, P152=0.00015374196620458413, P153=9.881244912765911e-05, P154=7.273237881940138e-05, P155=7.189250231738286e-05, P156=9.689151190717701e-05, P157=8.691388724372727e-05, P158=0.0001031125429313233, P159=0.0001103979899923095, P160=0.00013841237289729554, P161=7.094262869084754e-05, P162=8.362446117897546e-05, P163=0.00020118301220181615, P164=0.0004177363953544166, P165=0.00038016358031109625, P166=0.00046530686301563223, P167=0.00038014430896655445, P168=0.0001980579637810478, P169=0.00015898510619652813, P170=0.00036104996312795787, P171=0.0005210984812342428, P172=0.00032288618466362314, P173=0.000373652304479073, P174=0.0003102090790529595, P175=0.00017281766739832475, P176=0.0003311969791890607, P177=0.00032446996848019727, P178=6.522773760949425e-05, P179=2.435711506929038e-06),Row( P0=-524.3100549787783, P1=5.758074163396093, P2=-38.29525203887428, P3=-9.66016696919427, P4=-22.0297526176471, P5=-12.812722999648878, P6=-7.980515591374517, P7=1.5950547071208219, P8=9.51565408997977, P9=6.915283576602347, P10=14.171914514789323, P11=11.174367565942202, P12=-4.957645313104322, P13=-5.2946594516692205, P14=-9.673131414546978, P15=-0.6870127192750343, P16=-0.9168232141376189, P17=8.012877065474617, P18=1.0152678686018433, P19=0.6477523343470947, P20=3.041158604712437, P21=3.7161266622796725, P22=0.26478136649511913, P23=1.0821158385901002, P24=-1.2002098808441921, P25=-0.547907288388881, P26=0.33051814681593616, P27=-1.2466142144232697, P28=-0.912422974775202, P29=3.159197768735896, P30=0.8297981632502401, P31=-0.5072398727377772, P32=1.2246326243579575, P33=-1.6020802413504385, P34=-0.8224064040374943, P35=0.1512827292315415, P36=0.6843483625546695, P37=0.03983695515324215, P38=-3.2313994315960466, P39=2.566231936695209, P40=0.6594772477923294, P41=0.5886756357587671, P42=0.5564194935619335, P43=0.48991555410070914, P44=0.4511463325266503, P45=0.4874172228992087, P46=0.5262469449972123, P47=0.5312559546946786, P48=0.6191634324336749, P49=0.6666951154702604, P50=0.7098887808993891, P51=0.661762216029442, P52=1.90795951852608e-05, P53=2.3375224589117458e-05, P54=2.389907332731475e-05, P55=3.220512229462951e-05, P56=3.748252516666072e-05, P57=5.073066330470313e-05, P58=3.194918782308268e-05, P59=1.6962252775295573e-05, P60=2.3563093281802507e-05, P61=2.529810299885384e-05, P62=2.2775119068704164e-05, P63=2.647302490718219e-05, P64=1.9816637244347793e-05, P65=2.5299364728930407e-05, P66=3.385152413046501e-05, P67=3.17905890881535e-05, P68=2.352906013054138e-05, P69=3.7948130492231505e-05, P70=6.353512271099482e-05, P71=0.0005656814683135903, P72=0.00397077219522887, P73=0.010957300555549748, P74=0.040643807735079286, P75=0.10815338989067584, P76=0.1876120945920878, P77=0.4741111893949348, P78=0.6830961830114857, P79=1.0282800606572242, P80=1.4376540626430268, P81=0.9245302995396355, P82=0.4876694986628672, P83=0.22975024014757175, P84=0.09137107029821179, P85=0.025458621361601912, P86=0.003971016441001819, P87=0.0028771066914512187, P88=0.002473381871043963, P89=0.0015008129837064683, P90=0.0009075982786377979, P91=0.001990415998472757, P92=0.007968663956056466, P93=0.019908457369501992, P94=0.0349416319117803, P95=0.12455788026268874, P96=0.32926355849992417, P97=0.28912463664682325, P98=0.2086940640720541, P99=0.045458204559413386, P100=0.05425295237582599, P101=0.12187448270930282, P102=0.22678242932577172, P103=0.23036545432186586, P104=0.5323300307679789, P105=0.12428529836161048, P106=0.006747397763268339, P107=0.00325471734875172, P108=0.015952603086904694, P109=0.039289319003894124, P110=0.024466523957774315, P111=0.009866910537567419, P112=0.010457679550070697, P113=0.013733433248388943, P114=0.3343943638149964, P115=1.5674958660722933, P116=0.362695446430031, P117=0.09427759559550776, P118=0.0995934682548807, P119=0.07669779045629922, P120=0.059208755318113045, P121=0.06720728073142394, P122=0.02050948926733578, P123=0.01631344974055938, P124=0.04198451119605984, P125=0.04852314624904949, P126=0.07011585966855967, P127=0.003803283416396016, P128=0.0009092952642349467, P129=0.00511794690846343, P130=0.010910295230185132, P131=0.007803981250764744, P132=0.0032598050208703924, P133=0.001150553692048657, P134=0.0014533266172727655, P135=0.0025913076959872827, P136=0.010715281644674994, P137=0.0057895091413476835, P138=0.003076217715847113, P139=0.0013869303528236943, P140=0.0045181459762836974, P141=0.013006328201613071, P142=0.004314665408127447, P143=0.0017812350845309132, P144=0.0009476502912290144, P145=0.0019559260510266705, P146=0.0008281026658795193, P147=0.0008425969483443255, P148=0.003431674426777346, P149=0.005590220543430872, P150=0.0019717580535506497, P151=0.0011568037032674793, P152=0.001599499164310359, P153=0.0012431723370032782, P154=0.001121481080570735, P155=0.0009165291054645653, P156=0.0006540358461313006, P157=0.0009176956959354599, P158=0.0017201414803245873, P159=0.0015520910263809647, P160=0.0017210207765626295, P161=0.00204939326011152, P162=0.001575886294599252, P163=0.001178154738195833, P164=0.0011926376544321316, P165=0.0011761489172935837, P166=0.0006370485380962684, P167=0.0004071812080658299, P168=0.0004200314895293128, P169=0.0003991772440136475, P170=0.0004945039315634774, P171=0.0005807088037947333, P172=0.0006605862024279069, P173=0.0007075753296606586, P174=0.0003858056804767668, P175=0.0003480711577567295, P176=0.00025083871546688254, P177=0.0002029606901831147, P178=0.00030353952794255916, P179=5.56126769540693e-05),Row( P0=-681.631644315794, P1=42.371760083978785, P2=-19.789208720535797, P3=-0.23241643253848335, P4=-15.247628268315083, P5=-6.017128677973233, P6=-8.665671659154409, P7=-6.127614853701081, P8=-4.802136909004355, P9=-9.655023086595799, P10=-5.196012356454103, P11=-4.208626562831576, P12=-3.785152070455679, P13=-2.4770303212331672, P14=3.860547533634212, P15=6.952568599703226, P16=1.8635600004571369, P17=9.576558376717177, P18=5.250301992345013, P19=6.298380070158619, P20=2.7606644491442314, P21=2.393647950691577, P22=-0.14817898570878915, P23=1.2767329946206973, P24=-0.4445225775675814, P25=-2.682674354189699, P26=-3.4956983815139413, P27=-3.880129693092919, P28=-2.954082571344341, P29=0.17427249593737545, P30=0.45629102301615937, P31=-0.26606453677011377, P32=-1.7703148805095212, P33=-1.0044306266135143, P34=1.2661187805869039, P35=2.1464806166374046, P36=0.1749977271695642, P37=0.04061585047318928, P38=-3.080553183016602, P39=0.37303946752642053, P40=0.6131766991928, P41=0.5343463337365567, P42=0.5465411290805223, P43=0.4772681421479574, P44=0.47546217736300145, P45=0.47766907156893135, P46=0.5214309790919832, P47=0.551993734180344, P48=0.5218042323240131, P49=0.5479829154059167, P50=0.5717422595087197, P51=0.6088485119508806, P52=4.154159409340256e-06, P53=9.900496011696978e-06, P54=6.193709761874807e-06, P55=1.2888266682613532e-05, P56=2.6864659337157035e-05, P57=2.589711906743444e-05, P58=1.0598631928139996e-05, P59=3.824657909094777e-06, P60=4.330170853770656e-06, P61=1.5394221626208957e-05, P62=0.00129450504453929, P63=0.008419398915740808, P64=0.007275483060941475, P65=0.022014682436408424, P66=0.0747746209927005, P67=0.12427258436821291, P68=0.037719093772625965, P69=0.030099087338670102, P70=0.026314546209126786, P71=0.005599956238791602, P72=0.011984719598508472, P73=0.010345281539152122, P74=0.0017029537115129164, P75=0.002434655504117044, P76=0.0027726330491615265, P77=0.002875131544066403, P78=0.004756654468094117, P79=0.00970322691542521, P80=0.015300426908954518, P81=0.005880600189730946, P82=0.006998212003172153, P83=0.00877970197259164, P84=0.00333157513647017, P85=0.0010700948375652708, P86=0.004614135772270757, P87=0.03849075900948849, P88=0.03144788668620357, P89=0.002452288913797907, P90=0.0014199981548081758, P91=0.006548435293178763, P92=0.01343973186969998, P93=0.0033774928554657795, P94=0.005633801091631256, P95=0.00938303123086791, P96=0.004602355670585335, P97=0.0008188371913019396, P98=0.00040097908446371357, P99=0.00010057932114722226, P100=0.0007453347984117982, P101=0.006236849006018393, P102=0.002300934408597549, P103=0.0023073571244695747, P104=0.005871765764808949, P105=0.00357970269099137, P106=0.0017110804912315843, P107=0.0012814824262486005, P108=0.004668153976719246, P109=0.0013822000366240607, P110=0.011324668100517243, P111=0.006135948303404114, P112=0.002315469649472, P113=0.0010939575359556507, P114=0.000621225948130444, P115=0.0016944019393570092, P116=0.006000861395227176, P117=0.0016408431693210254, P118=0.0009630152293010832, P119=0.0006309124281630146, P120=0.0009173129366200058, P121=0.0016602021799530947, P122=0.0011741571317659236, P123=0.0008009194947744422, P124=0.00015519862391739488, P125=0.000639103947503314, P126=0.001809006278803356, P127=0.00023996445974429668, P128=5.223249270073717e-05, P129=0.0001401443803416899, P130=0.00012820931384064082, P131=3.39969492393274e-05, P132=2.5114650206642495e-05, P133=4.1583978600829635e-05, P134=9.72115903916157e-05, P135=7.490956330040198e-05, P136=4.349230983340201e-05, P137=0.00010241548386784834, P138=5.974801736085174e-05, P139=0.00011870735863140004, P140=0.00018008232487745323, P141=0.00010276874919012128, P142=0.00020663074079887436, P143=0.00011362124060012678, P144=4.729730496233171e-05, P145=3.986400971209157e-05, P146=3.0663275732143296e-05, P147=2.403092285720044e-05, P148=2.81645646288961e-05, P149=2.394123947797054e-05, P150=2.270793556005431e-05, P151=3.0783373388822214e-05, P152=2.7050184620659072e-05, P153=2.2062691320154144e-05, P154=1.5248258148955348e-05, P155=1.636798008249457e-05, P156=1.2033055358901834e-05, P157=1.2634715183732479e-05, P158=1.4223859630791654e-05, P159=1.5083123050069572e-05, P160=1.3786768937716754e-05, P161=1.3801331306409301e-05, P162=8.578147086895731e-06, P163=1.2053186781911286e-05, P164=1.5227037394929392e-05, P165=1.2443373561577528e-05, P166=6.02933435671811e-06, P167=4.047319533539253e-06, P168=2.538735597640823e-05, P169=4.543331248152076e-05, P170=4.205205656475715e-05, P171=1.803490800091621e-05, P172=5.975374271307406e-06, P173=5.8741534078121975e-06, P174=6.4166880667471665e-06, P175=3.766055116446678e-06, P176=4.55374896034508e-06, P177=4.81214282697438e-06, P178=3.978196532593386e-06, P179=5.78275768473765e-07),Row( P0=-590.0828411824283, P1=35.922960377481694, P2=-31.685554322593482, P3=-9.563854065579708, P4=-20.21779026951538, P5=-9.547298905973532, P6=-10.692786849237836, P7=-2.7934077488848104, P8=-4.33921036220263, P9=-11.490369024116212, P10=-8.226665028798871, P11=-4.12351549495411, P12=-10.35265369016827, P13=-0.31490923454455094, P14=-4.026386886365492, P15=9.860703745258784, P16=7.593728081983659, P17=11.83091468982211, P18=9.928045838391547, P19=3.45905847890845, P20=4.226007523388656, P21=-1.1900803895549108, P22=-4.124391208710438, P23=-0.1323800119072136, P24=-1.969275918350656, P25=-1.8909718217415452, P26=-3.4844572383819647, P27=-3.191804153074201, P28=-3.753741726950277, P29=0.5536059411090573, P30=-1.3168321138528702, P31=0.7730253178086444, P32=0.6313700587244044, P33=-0.23616191619163235, P34=-1.5970639727608062, P35=0.2828949772208621, P36=0.010703345507837752, P37=0.6994897923174325, P38=-3.0228273175059988, P39=-0.3228914624575307, P40=0.7335142629151385, P41=0.6659559437231639, P42=0.6512817195081072, P43=0.5681893919777415, P44=0.5868818033652761, P45=0.6146677913753342, P46=0.628045091758449, P47=0.6210079419917585, P48=0.6344294198306746, P49=0.6719171108164571, P50=0.734039543011197, P51=0.7593399208195145, P52=2.2316668975498835e-06, P53=5.23216352746388e-06, P54=1.0440225575900015e-05, P55=7.54018708470407e-06, P56=7.299973613970038e-06, P57=2.1632048135598198e-05, P58=2.8950396529469513e-05, P59=3.749065012315471e-05, P60=4.7978491775664957e-05, P61=0.00011035443685144224, P62=0.0015709514432087216, P63=0.009553280687247484, P64=0.0140161357604883, P65=0.04046664727189118, P66=0.1980751425616731, P67=0.32391019772131513, P68=0.09494171530673234, P69=0.06294192120092226, P70=0.04383783830870991, P71=0.004932927735206233, P72=0.0006655630812846189, P73=0.0005588322182727724, P74=0.0007442788089186817, P75=0.0009632534650698119, P76=0.001626533813383546, P77=0.0045694043798753855, P78=0.008812166330310813, P79=0.01855702976001132, P80=0.05302259291213751, P81=0.0989623330377307, P82=0.11721538165064924, P83=0.09340011771918989, P84=0.07294960728927131, P85=0.03839757829079741, P86=0.018009170741873384, P87=0.03407266309227209, P88=0.02507739198721214, P89=0.005558694838967071, P90=0.0034886272671636764, P91=0.007255118111903499, P92=0.027228203354159156, P93=0.024556975200246595, P94=0.041558941358026104, P95=0.05666133228926704, P96=0.03575105380461482, P97=0.029725199381238163, P98=0.04903008375983928, P99=0.055987959459445714, P100=0.04850644315768698, P101=0.061355354859389706, P102=0.032942072155624356, P103=0.06083389769178094, P104=0.11482992277441278, P105=0.03802316102541865, P106=0.019508844019256157, P107=0.022921123510244976, P108=0.026209937322409272, P109=0.01846770635493111, P110=0.029827755932696414, P111=0.005681329556049649, P112=0.008946249601141638, P113=0.03264521264539452, P114=0.04285910291657517, P115=0.08930732429421033, P116=0.07286704682117592, P117=0.009164799979686276, P118=0.007749767834572006, P119=0.004358067689632439, P120=0.004328897274200106, P121=0.004257420439987055, P122=0.00501165641457777, P123=0.003588303640887139, P124=0.002163426727909075, P125=0.0014817807001727284, P126=0.0020883903905519114, P127=0.0010797287634629542, P128=0.0007924897991463956, P129=0.0005648615261044857, P130=0.0011145119290176888, P131=0.0010918671889454388, P132=0.0010131911727558395, P133=0.0010086620021544075, P134=0.0005325468865693312, P135=0.0009272865255660642, P136=0.0004055953629486112, P137=0.0008994141489603093, P138=0.0009186044164922042, P139=0.0013822541459244002, P140=0.0013178750024445886, P141=0.001347467721781581, P142=0.0011944067196354591, P143=0.0005235527386117164, P144=0.0002626565020619081, P145=0.00023262098381195172, P146=0.00021246319808241468, P147=0.00035888276042439986, P148=0.0005515695208134166, P149=0.0013028680472083117, P150=0.000933797124863804, P151=0.000813211448523727, P152=0.0010002120720058758, P153=0.00048387203405012876, P154=0.00029469815793721737, P155=0.0003607293949279313, P156=0.00030754743885168224, P157=0.0002575733128187091, P158=0.00028175522497387586, P159=0.00034085792083907054, P160=0.0004705421829295728, P161=0.00017185365494859313, P162=0.0002521526278131524, P163=0.0004075346331731633, P164=0.0006104185632807186, P165=0.0011948629912753562, P166=0.0013859746357475593, P167=0.0014074779152435804, P168=0.0006011122072348925, P169=0.0003126150449988088, P170=0.00044930309624520675, P171=0.0005080565330784488, P172=0.0005416686179036324, P173=0.00036231229542625594, P174=0.0002488740644027515, P175=0.00026185547974474466, P176=0.0003133315282618355, P177=0.0003204979464866258, P178=0.00012168960888648593, P179=6.525314534600698e-06),Row( P0=-681.1382066246482, P1=34.4110084400702, P2=-21.48456665303299, P3=-9.393305038587833, P4=-16.6807383960455, P5=-10.916561940360346, P6=-15.033962967997738, P7=-7.6736203449121305, P8=-7.76341269503431, P9=-10.702254083224636, P10=-8.846136830506444, P11=-2.701575960033248, P12=-6.059502238804542, P13=-2.9083694561239493, P14=3.1582307810640518, P15=8.98989800165382, P16=5.090708323025494, P17=11.959965983169432, P18=4.456842702164465, P19=4.710608694949302, P20=1.565508093744228, P21=1.5983234996325857, P22=-1.7182412134221003, P23=-1.3312090481507461, P24=-4.030098854713502, P25=-6.865061779433113, P26=-3.0824729945503573, P27=-3.656725735403433, P28=-1.3220573501000286, P29=1.1618738813601537, P30=-2.3930145863174404, P31=-0.8955729596142634, P32=-0.8183599339818466, P33=-0.11805176620127575, P34=-0.5123351948692427, P35=-2.16445704082763, P36=-2.6120027044415637, P37=-0.8594206094634235, P38=-1.6848172257906662, P39=1.5785549128789602, P40=0.5870526103689955, P41=0.5620927164150245, P42=0.5294087316008632, P43=0.4759729798978556, P44=0.4293686148639294, P45=0.41087843041914074, P46=0.4500093635494921, P47=0.5135479547043936, P48=0.5582076109313922, P49=0.5863136407491187, P50=0.5763073151750248, P51=0.5682838000207895, P52=4.716003909668975e-07, P53=1.3611645322621525e-06, P54=2.0089321572925262e-06, P55=2.4357971416836146e-06, P56=3.5100356910460623e-06, P57=3.890215192014577e-06, P58=2.619471302580423e-06, P59=1.253755001055542e-06, P60=1.1369224081738927e-06, P61=6.144228363772866e-06, P62=0.0001537925649964537, P63=0.0020199069890128826, P64=0.007083865575662464, P65=0.019624892723197382, P66=0.03759984077684409, P67=0.056847671805642125, P68=0.03538356078772024, P69=0.03382224267960584, P70=0.02667193356237389, P71=0.005474595579235127, P72=0.0003562132724119991, P73=0.00016125920171431736, P74=9.764689255630047e-05, P75=0.00018268044111683832, P76=0.0013576904365224074, P77=0.0029656550245078283, P78=0.007065499988661728, P79=0.009339492490555778, P80=0.005543252267250831, P81=0.004271554957986913, P82=0.007107032926539529, P83=0.014858488937699692, P84=0.03980415447234581, P85=0.03656101275216877, P86=0.009616577325481252, P87=0.004621138232967775, P88=0.007109016092516554, P89=0.0041958207307252, P90=0.002742083345096516, P91=0.005333053508721324, P92=0.0014825398357065018, P93=0.001066182451467502, P94=0.009670538392466756, P95=0.012820686864401756, P96=0.0048194312708291515, P97=0.005027252550914312, P98=0.0057209913297015325, P99=0.004883378493928862, P100=0.008254263099541128, P101=0.004933157695386072, P102=0.001539966034375363, P103=0.006407306697008556, P104=0.013430134469759578, P105=0.007130165520508062, P106=0.008276721819307706, P107=0.005770911796985755, P108=0.0030561763104828125, P109=0.0015828563150571335, P110=0.00950604768765195, P111=0.004946862208549923, P112=0.0048024380014018275, P113=0.0018231988269105362, P114=0.001517339544015792, P115=0.009133243863084511, P116=0.015064797484044379, P117=0.0019564812641263453, P118=0.0009473714018575998, P119=0.0007235232715758134, P120=0.0011963168472154044, P121=0.0018478184688510162, P122=0.0005749575126536964, P123=0.00020214117439133907, P124=0.00044938766787060456, P125=0.0006629788496388552, P126=0.00015101279280954247, P127=8.27993592110505e-05, P128=0.00035573127155014904, P129=0.00010172848875790303, P130=4.916112011235447e-05, P131=6.595463918332632e-05, P132=3.083253308638995e-05, P133=2.3047334180656466e-05, P134=2.2757082724623964e-05, P135=1.410547700094723e-05, P136=2.7659366725198383e-05, P137=6.647806765305366e-05, P138=4.276643136528239e-05, P139=9.816716884535802e-05, P140=0.00017625505499666998, P141=0.0001726222301166166, P142=0.00017548637907242222, P143=7.633253759003006e-05, P144=4.4881452433893816e-05, P145=1.9449998900474515e-05, P146=2.366182058917811e-05, P147=2.371664213958706e-05, P148=3.3730273738823645e-05, P149=3.754403339541886e-05, P150=2.6872141837468436e-05, P151=1.3783390264000055e-05, P152=1.702661469356701e-05, P153=2.5218120386800655e-05, P154=2.469567137979022e-05, P155=1.7045147925884558e-05, P156=2.4889504103534864e-05, P157=4.753750657653772e-05, P158=5.796620308435929e-05, P159=7.423614851235779e-05, P160=0.00018823597897957384, P161=0.00018305362103931166, P162=0.00013611634103139953, P163=0.0002880519142962169, P164=9.631790674603128e-05, P165=0.00010511275828602152, P166=0.00011009838563780138, P167=9.775380108326768e-05, P168=5.445142515487493e-05, P169=8.796283733547977e-05, P170=6.655506465125049e-05, P171=3.6620422882456325e-05, P172=2.302809631255244e-05, P173=1.9835679807106487e-05, P174=1.353836292249526e-05, P175=1.2485315500785194e-05, P176=1.5604228969581303e-05, P177=8.409027943696092e-06, P178=6.601106268988169e-06, P179=1.4152509309605198e-06),Row( P0=-612.4715186958845, P1=33.26447789527439, P2=-30.60686068052737, P3=2.0378014793431802, P4=-19.1368178809331, P5=-8.823872416169616, P6=-8.486701583512087, P7=-7.4367485999030425, P8=-2.0357877545784393, P9=-9.117441402919917, P10=-2.9562186710582274, P11=-0.42003068403225324, P12=-2.178202305726852, P13=-0.005462366989300303, P14=-2.5878538237806947, P15=4.584480695187551, P16=-0.4387731127471351, P17=4.829623600524278, P18=2.502797348407669, P19=1.5437858627456156, P20=0.7303075686389795, P21=0.3751702492984731, P22=-2.070224629669236, P23=0.21839587770089758, P24=-4.460314283559733, P25=-4.080318094078399, P26=-2.387790867471152, P27=-2.576060465418216, P28=-1.6225573749485045, P29=-0.9785045473786395, P30=0.6779373657650981, P31=-0.28448225681697037, P32=-0.028575076766003897, P33=0.1193710586474123, P34=-0.5199958828241995, P35=-1.805034995899494, P36=-2.5690961935691212, P37=2.3947521582248683, P38=0.5537347510661284, P39=-1.0886139803006556, P40=0.6752611989497459, P41=0.6573293583134683, P42=0.6576673625054175, P43=0.5998034862371818, P44=0.5818353117730575, P45=0.5851320421990359, P46=0.6040229458082134, P47=0.6700165855152752, P48=0.7463472550445339, P49=0.7228375057409764, P50=0.6987142466555565, P51=0.6728984995159553, P52=2.1739029174533417e-06, P53=3.820446810724993e-06, P54=1.6941181304372146e-05, P55=3.804156356173992e-05, P56=4.984246716017985e-05, P57=6.836223027267237e-05, P58=5.488281030331959e-05, P59=5.556892210898285e-05, P60=6.044765835429306e-05, P61=9.010554396135124e-05, P62=0.0014057210453122433, P63=0.006081311276865963, P64=0.010958625656282034, P65=0.032910981254388655, P66=0.0456629388089059, P67=0.036160613225099364, P68=0.06745395402492883, P69=0.07612050145818558, P70=0.02721603448618886, P71=0.013296095278017225, P72=0.03264385853036875, P73=0.05092172205024162, P74=0.01897854464745809, P75=0.008967643798186804, P76=0.019716008836135376, P77=0.17163025600637863, P78=0.2581740054941461, P79=0.07877648198505859, P80=0.037938554149377496, P81=0.026160407350119277, P82=0.012104423494180689, P83=0.007044712798674024, P84=0.011050819444266426, P85=0.009967291566292015, P86=0.012771781128070937, P87=0.008924053413537018, P88=0.007267529842212859, P89=0.007479429262387834, P90=0.009833223374783956, P91=0.01758772207406259, P92=0.04661173964857612, P93=0.033277144392649304, P94=0.01341917752576049, P95=0.023185651340330838, P96=0.02347654782468131, P97=0.016485401503161898, P98=0.015871449008782106, P99=0.007633008459814277, P100=0.011385910154597072, P101=0.04720450053977529, P102=0.0204127046617265, P103=0.004559344042497113, P104=0.008908114897782258, P105=0.020957988842655, P106=0.02088692555793715, P107=0.03671460408710523, P108=0.05971767571142999, P109=0.02145349657662787, P110=0.023333213090435942, P111=0.045783450150947945, P112=0.107218615877781, P113=0.05113339443392404, P114=0.02408352334459176, P115=0.018344700320781795, P116=0.01062474831787454, P117=0.04416730465239531, P118=0.06889494899394484, P119=0.04748123564109273, P120=0.014811950818124036, P121=0.010689250343263334, P122=0.02459651147974939, P123=0.013040766644919634, P124=0.0024378564930962287, P125=0.0014618152521054578, P126=0.009614096878455849, P127=0.011417173661523315, P128=0.000975590109470701, P129=0.0019426475733057707, P130=0.0029213199281672335, P131=0.001071929769256336, P132=0.001394570906897346, P133=0.0011212074601417669, P134=0.0005450584399733211, P135=0.0026622115912302837, P136=0.0011907562095850584, P137=0.003434489430753179, P138=0.0017719102123780266, P139=0.0025225850803240967, P140=0.0027553865852011786, P141=0.004207307350550255, P142=0.004438607173166619, P143=0.0015478287747450164, P144=0.0010116954688208434, P145=0.0009348623601502579, P146=0.0004608301946932593, P147=0.00070446474906962, P148=0.001157566710864429, P149=0.0007229236997453597, P150=0.0007845389822374757, P151=0.0006845081720309865, P152=0.0010283264447652528, P153=0.0006764472042682596, P154=0.0004992523807803957, P155=0.00022379197681544332, P156=0.0001840444967873276, P157=6.340680059767863e-05, P158=4.861934349905337e-05, P159=4.7125740457819713e-05, P160=5.396799311314957e-05, P161=3.16351753112108e-05, P162=2.1125109663782302e-05, P163=2.2616226146294374e-05, P164=2.082104074794806e-05, P165=2.519440881813216e-05, P166=1.328351369119035e-05, P167=1.0343153772300732e-05, P168=1.7566940788654558e-05, P169=2.3119009529298947e-05, P170=2.822956041567421e-05, P171=3.2422929453059685e-05, P172=2.3424769627992756e-05, P173=1.9993679018028707e-05, P174=1.9999417687716885e-05, P175=1.707704927201379e-05, P176=1.1422151286770258e-05, P177=1.4936117572564573e-05, P178=8.627960903471896e-06, P179=8.645586784894162e-07),Row( P0=-680.1650919562915, P1=39.79015132908858, P2=-18.92461768013698, P3=-5.545536851133655, P4=-14.635214807587943, P5=-9.35246083296882, P6=-9.925351237291462, P7=-6.299038791234062, P8=-6.423885184150608, P9=-9.151080775637846, P10=-7.378998957613352, P11=-3.1811745224470678, P12=-5.322714543527996, P13=-0.6804498561838869, P14=2.2427363344047957, P15=9.066917317194404, P16=4.974403589901787, P17=14.05613581057952, P18=7.899764934123038, P19=5.24068323253635, P20=3.847789003987599, P21=2.576279984942416, P22=0.6840089232336081, P23=2.149289616752797, P24=-1.4457950381643174, P25=-3.700727638146543, P26=-2.930164194451776, P27=-4.975729388659684, P28=-2.3376026778962093, P29=0.37072789558215347, P30=0.9814393760670688, P31=-1.4332818248974535, P32=-1.8718892020478612, P33=-0.9288771453876866, P34=0.17042029192367109, P35=0.32022481964748634, P36=-1.6689314493923948, P37=-0.6328589695621755, P38=-2.201401612569989, P39=0.34771674014713067, P40=0.5508445633181923, P41=0.4940605037567843, P42=0.47872821579770536, P43=0.43622460151588865, P44=0.4138608625421622, P45=0.40626522675366095, P46=0.47928083326626847, P47=0.49345634844965547, P48=0.5188000947055404, P49=0.5066341664744025, P50=0.5327370096245836, P51=0.5754957123419523, P52=1.517294734260167e-06, P53=4.497249076520461e-06, P54=4.432819832357864e-06, P55=3.106547215603529e-06, P56=3.4719377782844843e-06, P57=3.0883399626988408e-06, P58=2.297355636793788e-06, P59=7.867873736001746e-07, P60=1.8794613256168125e-06, P61=1.3301542778286958e-05, P62=0.0012495009329079104, P63=0.005841782064396467, P64=0.008007721646629367, P65=0.022602588816869034, P66=0.07076021296907556, P67=0.13658420687134815, P68=0.05783867971444261, P69=0.01504577240449248, P70=0.011202186362525039, P71=0.0028692750333743494, P72=0.0002468681643003027, P73=0.00013338776819373896, P74=0.0009125302776212818, P75=0.0024743406377355517, P76=0.0017366477770614556, P77=0.004017450354827182, P78=0.01239349453875082, P79=0.012807878966866026, P80=0.007143700014896657, P81=0.007551535639528985, P82=0.012399173172196389, P83=0.02145436168931549, P84=0.027657305534358686, P85=0.02457738418594516, P86=0.010040014054327894, P87=0.005674471377093718, P88=0.0027715274901678835, P89=0.0020769663223388856, P90=0.0054079101796345, P91=0.007269116956719023, P92=0.001515859164895548, P93=0.004438937847109877, P94=0.012597261125190877, P95=0.020167830410610123, P96=0.011029267208601354, P97=0.00679395923626432, P98=0.005023965824633326, P99=0.004166021795662958, P100=0.0018967385542978308, P101=0.0021852806407032864, P102=0.006081745641975132, P103=0.006158938695468623, P104=0.00908425729707539, P105=0.005079852841535586, P106=0.005973642438594751, P107=0.0017166249777164114, P108=0.0018259986318261492, P109=0.00999605114846596, P110=0.008004852699503993, P111=0.004270537504181219, P112=0.004347795206784622, P113=0.0011021148793168567, P114=0.0014051166409962184, P115=0.005728137662893976, P116=0.005094319511699367, P117=0.001968508736918077, P118=0.001323741380108276, P119=0.0038217629513959215, P120=0.0036157374472057047, P121=0.002873755438349908, P122=0.0011419960369168914, P123=0.0008371116530971913, P124=0.003692428708189822, P125=0.0008528343354149916, P126=0.00018912548719597633, P127=0.0005816508719701513, P128=0.0008989180564332413, P129=9.030889036757468e-05, P130=0.00023077807482321786, P131=0.0001850348029495319, P132=5.2751565847577484e-05, P133=0.00012154909788357027, P134=3.159024673569932e-05, P135=0.00014626193248743003, P136=6.380147518175342e-05, P137=0.00010712753734076483, P138=0.00020071158657218022, P139=0.0001973511483341046, P140=0.00036295502802394816, P141=0.00017738642655073814, P142=0.00017623047035272125, P143=0.00010241938911767003, P144=6.480153594943027e-05, P145=3.121333659944045e-05, P146=1.8652680997009052e-05, P147=2.3467722151726826e-05, P148=2.556909404354759e-05, P149=3.45060077524546e-05, P150=3.7974436569016966e-05, P151=2.3557680659653232e-05, P152=2.5956669454628363e-05, P153=3.916913139002106e-05, P154=3.3691991301828845e-05, P155=2.992132766109877e-05, P156=1.984011506841803e-05, P157=1.767349834651554e-05, P158=2.6155231273853485e-05, P159=3.478192118409643e-05, P160=5.2773018427702436e-05, P161=9.935659313968566e-05, P162=0.00016365298640452007, P163=0.00011400493293638814, P164=8.83891121653778e-05, P165=5.0655721894256173e-05, P166=5.381585939768422e-05, P167=8.003229417741662e-05, P168=0.00011473034849733805, P169=0.0001100537374238079, P170=0.00011885750898307836, P171=9.323517064232694e-05, P172=4.1529201938911436e-05, P173=4.850564858056017e-05, P174=3.413514301939135e-05, P175=2.7196665299970187e-05, P176=0.00010951024454929551, P177=5.583870956753853e-05, P178=4.826560138302601e-05, P179=1.2530909517572196e-05),Row( P0=-561.0308262702941, P1=5.0279129551442665, P2=-43.48328014068159, P3=-7.159564062932505, P4=-25.86040562724946, P5=-11.410305771854915, P6=-7.5803437243875536, P7=-2.113434315598375, P8=9.638766303467339, P9=6.889081252808976, P10=11.88361115750433, P11=8.776620925838614, P12=-8.527912441124919, P13=-9.655559991382711, P14=-10.615208576936265, P15=-0.17673922206165224, P16=-0.5021167126264112, P17=7.775553946738918, P18=2.001216816558807, P19=-0.18482717101765195, P20=2.355877802915446, P21=3.062440983660342, P22=-2.8955642404864776, P23=-3.7710518859546407, P24=-4.610229858892916, P25=0.18651107212244775, P26=2.855917891207037, P27=-0.48536615758898694, P28=-4.0528922441243065, P29=1.2138525187519547, P30=1.6423749424962912, P31=-1.0615127181329227, P32=-0.4193652323690334, P33=-0.1523123537614384, P34=-0.6816094515819794, P35=-2.00135126373158, P36=1.4124048320485447, P37=3.795970544440486, P38=-1.8729089935010141, P39=-1.9399317643033038, P40=0.6230589040396668, P41=0.5923386688844818, P42=0.5355141845659761, P43=0.47728437248431127, P44=0.45170068972950683, P45=0.44708731429017995, P46=0.44352323668698174, P47=0.48888422259779546, P48=0.5768487078019988, P49=0.7126907597673154, P50=0.7635403771085666, P51=0.6442101960041315, P52=2.343354690190663e-06, P53=3.1144403600467436e-06, P54=1.0199732390164249e-05, P55=2.451169963194262e-05, P56=2.4621987128747565e-05, P57=1.7721603465937904e-05, P58=1.3223370857782775e-05, P59=1.6957115770150645e-05, P60=1.8431003093331994e-05, P61=1.6978637190269307e-05, P62=2.341900315588287e-05, P63=2.338224608715453e-05, P64=1.5795705107805877e-05, P65=2.8400961825226936e-05, P66=4.556185904119792e-05, P67=4.851800435407639e-05, P68=3.242049780418701e-05, P69=4.122001718048001e-05, P70=5.392029599646559e-05, P71=0.0001253730233386829, P72=0.0004153011152410527, P73=0.0022403653682505193, P74=0.0204808236683811, P75=0.043142112408264696, P76=0.058384445811814734, P77=0.21087134063337926, P78=0.69541280840352, P79=1.2103749594836661, P80=0.7911980665374446, P81=0.3621288561218885, P82=0.34442315884017305, P83=0.24976072577394118, P84=0.34751122307269455, P85=0.2647923393587377, P86=0.03006203422534456, P87=0.00257459037239604, P88=0.000710554674217721, P89=0.0003363951352957108, P90=0.0002663032473187644, P91=0.000279471695989717, P92=0.00035035288109062874, P93=0.00031045946595062885, P94=0.0008501426957850712, P95=0.023742826938262976, P96=0.09432456425040744, P97=0.08469716075072005, P98=0.09728353341579112, P99=0.12012739347932948, P100=0.5093743829984666, P101=0.5075384806671033, P102=0.22224994270263795, P103=0.024470013653053305, P104=0.03483795804228452, P105=0.00898320058947803, P106=0.00494588672654862, P107=0.00922773481877242, P108=0.2945691260600411, P109=0.45772520868262845, P110=0.38669042597531955, P111=0.16090519883761922, P112=0.09188106383264921, P113=0.0818495878315077, P114=0.043032197442975845, P115=0.01712836710722393, P116=0.0065099296620168825, P117=0.05363109015789731, P118=0.12253101372135203, P119=0.11441312826540931, P120=0.02629774059503431, P121=0.15075708377194993, P122=0.09260435773246682, P123=0.009322913801168778, P124=0.002484458731471376, P125=0.0017445050318156295, P126=0.0037856215307125753, P127=0.008132219946618155, P128=0.0292130515424411, P129=0.010560144682511758, P130=0.0038117909479929487, P131=0.0010978672976744575, P132=0.0016080576509051446, P133=0.005141429891652947, P134=0.0041217335929524986, P135=0.002289067014416355, P136=0.0010912677803623105, P137=0.0015684975054069484, P138=0.005495058933875474, P139=0.007377314732154349, P140=0.004912061638936294, P141=0.001698780239229808, P142=0.0021141584548058706, P143=0.002177109667606749, P144=0.0014007349599421232, P145=0.0007883124869898241, P146=0.001091431271349834, P147=0.0009110305747042952, P148=0.0014445907964351179, P149=0.0034370855534363765, P150=0.001532298212691685, P151=0.0019616352655076608, P152=0.0039097468807690805, P153=0.004382242900382431, P154=0.0014764128911314708, P155=0.0005864823275763619, P156=0.00033087057684263056, P157=0.0002829164032975517, P158=0.0002522475670350578, P159=0.00016658181568515577, P160=0.00023979370377703142, P161=0.0002402173039068501, P162=0.0002294570822425422, P163=0.00012959557125870541, P164=0.00012751037385024517, P165=0.00014381878055106252, P166=0.00010243729810532333, P167=7.357853894948948e-05, P168=4.6497693990927034e-05, P169=5.1830101645506055e-05, P170=9.20364951158186e-05, P171=0.00010171662802838779, P172=7.03667420527761e-05, P173=6.644585279930301e-05, P174=6.377341248782584e-05, P175=3.10502781959009e-05, P176=2.7096095584204545e-05, P177=2.4449151682750574e-05, P178=1.879350590063894e-05, P179=4.5695407525370405e-06),Row( P0=-600.51600949859, P1=19.446619925784194, P2=-37.28496430079907, P3=-3.6712922035840854, P4=-24.575054345116694, P5=-11.14780128875973, P6=-10.582327658880155, P7=-7.75626665671803, P8=-2.473379847641833, P9=-7.74451720406387, P10=1.3028941086950063, P11=5.576981420212362, P12=10.017807999468243, P13=12.635409611205416, P14=9.216113364059728, P15=3.7148006674549943, P16=-9.114371691766566, P17=-3.5557921775316585, P18=-2.999815170650899, P19=0.036753352573789345, P20=-4.191271329957777, P21=-3.11450059330387, P22=-4.888964271043771, P23=0.7894572848003719, P24=0.3196471941563703, P25=-0.14093115926240204, P26=-0.7329975667603339, P27=-1.1854806724857678, P28=-1.8850189907230104, P29=1.4729432255913721, P30=-0.7333788357188009, P31=-2.1488055572325018, P32=-0.38054081572073584, P33=-0.891677270700685, P34=-1.6647166211615878, P35=-1.6924877347856986, P36=-0.7802010980948739, P37=2.235738062931641, P38=-1.9417600425398727, P39=-1.0631726077463646, P40=0.53838463628198, P41=0.5726769119274088, P42=0.5903008658714954, P43=0.5494241074517355, P44=0.5406022526127103, P45=0.5839738646312501, P46=0.5481759920614422, P47=0.5389619919830574, P48=0.48762567039250826, P49=0.5311764070768389, P50=0.5473365729600764, P51=0.4881819482035887, P52=2.8276532126713555e-06, P53=5.171713703024931e-06, P54=1.3212352419460034e-05, P55=1.9743399984715543e-05, P56=2.336622798448639e-05, P57=2.494407824600433e-05, P58=1.111590835011474e-05, P59=8.680740066676317e-06, P60=1.068336160194643e-05, P61=1.0603355946578452e-05, P62=1.3437218941984732e-05, P63=1.5515249372364548e-05, P64=1.9468584248668976e-05, P65=4.859329582316934e-05, P66=9.085996236275226e-05, P67=0.0002119943198722586, P68=0.003977661329744948, P69=0.02417102471920304, P70=0.050666559672171686, P71=0.054121347604410446, P72=0.16213038333080157, P73=0.1791865298218756, P74=0.24627170669941165, P75=0.29264054475131507, P76=0.08966446771097412, P77=0.01673631595477277, P78=0.004271942230446731, P79=0.0015563122209536798, P80=0.0007019920283410742, P81=0.000378275090135138, P82=0.00024215686354306756, P83=0.00026791310483939524, P84=0.0003090308583121304, P85=0.00028956146065257785, P86=0.00039078603653462874, P87=0.005685096630935593, P88=0.044594435303181405, P89=0.04455478643185086, P90=0.06539793583335676, P91=0.165748073828855, P92=0.3789667883219783, P93=0.10077642928342258, P94=0.05582197466925403, P95=0.0386983133616907, P96=0.013644435278090632, P97=0.0012021316043261824, P98=0.0003758985823147588, P99=0.0005305822830722152, P100=0.0265514489921933, P101=0.04654686655659065, P102=0.014999155308138716, P103=0.00571663592964311, P104=0.021118679379851322, P105=0.11726763883211005, P106=0.10938001084272411, P107=0.03608984185177554, P108=0.0060091619368345, P109=0.0016798050342497596, P110=0.0019300474771631838, P111=0.0057522290968481755, P112=0.004617904884291913, P113=0.03565587544887484, P114=0.037833014505820085, P115=0.01367380295585927, P116=0.009551395677850322, P117=0.005954960701347754, P118=0.0012352497505948774, P119=0.0016350073002684138, P120=0.005564205670524737, P121=0.012453807330797982, P122=0.013501747744972398, P123=0.002443352496347761, P124=0.0008749760518659353, P125=0.0015282655032275573, P126=0.0033565789088042, P127=0.0025992390875948664, P128=0.0003603866040242421, P129=0.0015303204820317398, P130=0.0029122800123064702, P131=0.0013115519597243798, P132=0.00036705707720315127, P133=0.000362121372847777, P134=0.0011221902219278766, P135=0.001381022382744258, P136=0.0010082995397129463, P137=0.0011642212584052505, P138=0.003410591105314298, P139=0.0017664905312313294, P140=0.000931627783399333, P141=0.0022591761210568355, P142=0.0018368525151566898, P143=0.000606839352014523, P144=0.00040765456745022426, P145=0.0002952199487732718, P146=0.00022960062773530183, P147=0.00033874828624719857, P148=0.0004358710098069144, P149=0.0004651269474346333, P150=0.0005044733030054024, P151=0.0005714673515424731, P152=0.0005195349136707703, P153=0.000785716343242763, P154=0.00038874600637849456, P155=0.00025956406348102566, P156=0.00018725844712697692, P157=0.0001699599227718246, P158=0.00020596436375536678, P159=0.00032143724179058863, P160=0.00020839775597642922, P161=0.00015734000460132668, P162=0.00014507539999024922, P163=9.479948856065521e-05, P164=0.0001066422933812719, P165=9.734458439915454e-05, P166=7.117035197355804e-05, P167=7.794881805098704e-05, P168=5.002332999350597e-05, P169=4.335177684218696e-05, P170=5.688974799227901e-05, P171=6.44379614151915e-05, P172=4.791896938319728e-05, P173=4.3408235902102453e-05, P174=2.9095977980389738e-05, P175=2.2906813360188846e-05, P176=2.23438733288731e-05, P177=2.0586029446144524e-05, P178=1.271813446390123e-05, P179=2.332562860713477e-06),Row( P0=-574.7405299705961, P1=27.412143270467595, P2=-35.24005802620317, P3=-3.9049467075004998, P4=-23.418332567369006, P5=-16.482011164992734, P6=-9.08998168508335, P7=-7.541990081198151, P8=-7.267329574270541, P9=-12.367716718918485, P10=-1.6657923421972562, P11=2.134599533935578, P12=-2.3111892194800925, P13=7.333894869500307, P14=0.5197686438159763, P15=8.101410470911915, P16=-0.3703829003486277, P17=6.134082646663296, P18=6.373319715377077, P19=3.5850082282703446, P20=5.897698785442776, P21=2.0429082368145495, P22=-0.6326816283104193, P23=1.2537627725153369, P24=-2.2661569758031486, P25=-4.020619762919926, P26=0.01563218859063691, P27=-0.3590748644311968, P28=-2.0934465020962856, P29=-0.8352853703174699, P30=-1.0144996653609304, P31=-0.10096185982308717, P32=0.2789167589913277, P33=-0.2563937957739235, P34=-3.050840671077867, P35=0.4538341930870617, P36=0.9618510005049542, P37=1.4585729647771495, P38=-1.8598414134105403, P39=-0.7837694321017891, P40=0.5801610980941724, P41=0.5609803657275745, P42=0.6139458226127633, P43=0.5404593937997841, P44=0.49857397009956, P45=0.5128441895024484, P46=0.5390425874034609, P47=0.6083946867547413, P48=0.6066204302827994, P49=0.6285409242665703, P50=0.677066979891433, P51=0.6180558852658024, P52=3.2430679790562747e-06, P53=8.22012105781133e-06, P54=6.112713691184521e-06, P55=3.830135531406536e-06, P56=6.124583238464217e-06, P57=7.24897093815518e-06, P58=5.630821407186112e-06, P59=3.3968396084894765e-06, P60=6.010176638651673e-06, P61=1.858483073597106e-05, P62=0.0007226265431595446, P63=0.004809050308517529, P64=0.010875216014096818, P65=0.04885388421130038, P66=0.07346421774278204, P67=0.06817929445003727, P68=0.10847105113789927, P69=0.42460934902031894, P70=0.4224367327741005, P71=0.138401377057133, P72=0.057126706772520755, P73=0.09331147458321824, P74=0.17766193788961823, P75=0.2161267111656588, P76=0.18183463180387643, P77=0.051280784743070996, P78=0.020273180671380182, P79=0.03673749677913321, P80=0.056098323885797975, P81=0.03769462875434314, P82=0.031928462191838626, P83=0.03925917436015203, P84=0.02769549452428071, P85=0.01568179660796244, P86=0.05255042652724976, P87=0.12419869288183394, P88=0.1675789161562098, P89=0.1277590812510053, P90=0.08065174023224789, P91=0.08908983763706343, P92=0.1090165866339369, P93=0.09713776527513865, P94=0.13175337351095578, P95=0.10026885697486428, P96=0.060927851904944, P97=0.021612226331660774, P98=0.04265944426441775, P99=0.04945724132284268, P100=0.11703088535923493, P101=0.2025139911234529, P102=0.08616697679286171, P103=0.04360300896946237, P104=0.07546436608215329, P105=0.030840748077113483, P106=0.03490541715712006, P107=0.44733891564840456, P108=0.3581346294938855, P109=0.07998469283832284, P110=0.06704530837843795, P111=0.03716756393945857, P112=0.03261233352798393, P113=0.015769508986410545, P114=0.019238821951371304, P115=0.028971506400151024, P116=0.01892863762479028, P117=0.026184499214875846, P118=0.05621123057627606, P119=0.15543020137892852, P120=0.17411099085737233, P121=0.07785453405569145, P122=0.0627910439593627, P123=0.027701006962977587, P124=0.013645870496651455, P125=0.013622471997843883, P126=0.020959181731062433, P127=0.0060355568840880745, P128=0.0030096053463235946, P129=0.00860015705283788, P130=0.009178008633177807, P131=0.006465843422556579, P132=0.004040798398682095, P133=0.003009895676817667, P134=0.0027934707734712213, P135=0.0021410395811199854, P136=0.0032684822508254954, P137=0.007482247699495003, P138=0.0056987487262501815, P139=0.00871416798245387, P140=0.010659631298927465, P141=0.01226173108805178, P142=0.003119998245346935, P143=0.0021973168029366247, P144=0.0010719180974996172, P145=0.0009691826910489838, P146=0.0011736311754496566, P147=0.0015454200441206447, P148=0.0022419223181925135, P149=0.003026141314342386, P150=0.0034831878825558687, P151=0.0021980285868173285, P152=0.0024263065372731583, P153=0.002236584178999772, P154=0.001783839588853081, P155=0.0016474307275603152, P156=0.0006136421622077301, P157=0.00019385198481114446, P158=0.00013701134660904437, P159=0.00012641705996051798, P160=0.00013577207336149688, P161=0.00014443601203393812, P162=9.809831160096822e-05, P163=0.00010986179559925745, P164=0.00018130519945827142, P165=0.00020220562395753132, P166=0.00018154611364984454, P167=0.0001118772671660986, P168=9.046973973364624e-05, P169=0.00016078024610042166, P170=0.00015259938946947296, P171=0.00013324272962332196, P172=0.00010328945417372999, P173=9.384375105531678e-05, P174=0.000101372001487388, P175=6.596779993555685e-05, P176=4.644532591679077e-05, P177=6.571170642766075e-05, P178=3.5763752912034994e-05, P179=4.677729730982409e-06),Row( P0=-602.3208164487614, P1=38.7317625242326, P2=-24.796343700779314, P3=-4.448933949678252, P4=-18.49011390476625, P5=-3.7792930591357847, P6=-10.78959921628677, P7=-5.443872394193863, P8=-3.494824658294588, P9=-11.000738540611954, P10=-8.301965425964374, P11=-1.8174131098835549, P12=-5.425375336238629, P13=-0.05052225159788459, P14=0.6224222810199408, P15=7.1771324150132125, P16=2.625323646458265, P17=7.21919672502876, P18=2.3350428812265767, P19=0.33058813217934674, P20=-1.0667212072744598, P21=-0.8888988385773625, P22=-2.1172091301421467, P23=-0.28074054051347447, P24=-2.8999626313075653, P25=-4.6485142436682185, P26=-2.994490397294522, P27=-0.7718647312969557, P28=-1.7472299513856622, P29=0.2601084111452296, P30=-0.23649383945460264, P31=0.33424437823534625, P32=-0.18107763685364128, P33=-0.4033848502020069, P34=-1.199665909663015, P35=-0.26204481296478677, P36=-1.4487509876124824, P37=0.8010371066525104, P38=-2.4315389540012875, P39=-0.3689041097618061, P40=0.7174324436736488, P41=0.7030105902451182, P42=0.6869947705070517, P43=0.6268691650262598, P44=0.634432474031656, P45=0.6440177370489236, P46=0.6844063570580641, P47=0.693251233919323, P48=0.6846868114537177, P49=0.6607864717375791, P50=0.6883276206253532, P51=0.7201404717366607, P52=2.356528272294023e-06, P53=6.438884053739779e-06, P54=1.5041681967200422e-05, P55=2.37383035338198e-05, P56=3.238708098889072e-05, P57=6.318856260246844e-05, P58=6.773630559711336e-05, P59=6.603797518847148e-05, P60=6.699155629183505e-05, P61=0.00023558121804261367, P62=0.003892081638474209, P63=0.009028283770891399, P64=0.007836831110733524, P65=0.017598350378354857, P66=0.05085754785572197, P67=0.08095636548442774, P68=0.07312311091188707, P69=0.1715718395715098, P70=0.24537132198118533, P71=0.06427567506609876, P72=0.004391128152589391, P73=0.0026178760154058466, P74=0.0026932593039266195, P75=0.0024270218966244536, P76=0.0012686219328913075, P77=0.0019032792901427721, P78=0.002640042724044686, P79=0.003942738517403877, P80=0.011402967249715127, P81=0.024328421461743056, P82=0.029504378937694156, P83=0.023171319359668062, P84=0.018718951100314102, P85=0.015271230828762325, P86=0.04551025664402239, P87=0.05756001917636676, P88=0.03742684901223022, P89=0.01405698747967872, P90=0.004093830198350355, P91=0.004643794094724778, P92=0.008531751899414382, P93=0.010209511575878145, P94=0.030156807082365917, P95=0.056352861963654695, P96=0.03164130342639233, P97=0.024538603120979274, P98=0.029491647546107457, P99=0.009766408584314613, P100=0.00461809325467974, P101=0.006873910773682771, P102=0.01516835472312829, P103=0.019057914483475203, P104=0.029304493651806814, P105=0.011785796412744125, P106=0.01042832966369877, P107=0.012819180025803265, P108=0.02187941754918004, P109=0.08324796929904234, P110=0.06576435753426706, P111=0.05254789075707727, P112=0.036027084150111396, P113=0.01438241733254886, P114=0.01999345925157278, P115=0.03823174743278881, P116=0.03310067264341565, P117=0.01380063548399943, P118=0.007754549596170053, P119=0.012443399817669116, P120=0.005942767951793979, P121=0.002129321518030892, P122=0.004139053593436857, P123=0.007343067279626642, P124=0.002928274982574657, P125=0.001156366578403208, P126=0.0034526764953758215, P127=0.0014705950459493614, P128=0.0009031170424479873, P129=0.0013287378520161737, P130=0.0006449336017995436, P131=0.0004412178705641471, P132=0.0005798893572987173, P133=0.00047218244622862396, P134=0.0003072263683857065, P135=0.0006359361776956046, P136=0.000477329734654895, P137=0.000654847638680204, P138=0.0008922110179455014, P139=0.0009249086069002897, P140=0.0012482041492100759, P141=0.0011056780822798237, P142=0.0008742923492311233, P143=0.000447392217319418, P144=0.00042193148641015067, P145=0.00022054612545207688, P146=0.0002092313848205656, P147=0.0002392016312800443, P148=0.00033666845750934246, P149=0.000391802660884547, P150=0.00040277661593951666, P151=0.0003475524913189355, P152=0.0004436092472148848, P153=0.00024515410993369163, P154=0.0001924345753611339, P155=0.00021429851146210718, P156=0.00019819909191033961, P157=0.00011816141775673775, P158=9.234169838974522e-05, P159=0.0001226439415860928, P160=0.00010535889153890828, P161=8.887625400153268e-05, P162=0.00010198885297765072, P163=0.0001523635047133156, P164=0.00018044361785873156, P165=0.00019353161473649124, P166=0.0001254129626458876, P167=5.438607614785502e-05, P168=4.907275681356379e-05, P169=4.478631015525075e-05, P170=4.2028516098604084e-05, P171=4.2475782098654146e-05, P172=2.3098207655152393e-05, P173=2.3448299010938984e-05, P174=4.108829558749857e-05, P175=3.0205880565433524e-05, P176=1.7110075249383355e-05, P177=1.5272730425890235e-05, P178=7.814881531568306e-06, P179=1.1531788440231942e-06),Row( P0=-666.2972869161026, P1=38.5630400969016, P2=-16.980918487851522, P3=-2.9674507266653656, P4=-16.25812960259249, P5=-6.9741161720951625, P6=-8.255167452767786, P7=-8.519650062288044, P8=-6.161715735286105, P9=-8.705025730695624, P10=-8.07966188684209, P11=-3.1662386436768677, P12=-4.695187997244134, P13=-3.603962140823392, P14=0.4551924293178204, P15=2.169830494858773, P16=0.6801476814716929, P17=6.483148037615854, P18=5.128053972668286, P19=7.062637635321536, P20=4.662814831033688, P21=4.8261369935171405, P22=0.5624937548229171, P23=-0.4573051858420837, P24=-2.9781761175015222, P25=-5.448246529930634, P26=-2.9571008904847442, P27=-2.619833396386084, P28=-1.2360811412916575, P29=-0.5576669281712299, P30=-2.512602148075007, P31=-2.4076981645318685, P32=-1.937670006900691, P33=0.5172213614687597, P34=-0.4301865679697886, P35=-2.1493289294659417, P36=-2.8535582247454703, P37=0.7543585889950365, P38=1.6669763850735497, P39=3.7222213308838694, P40=0.6319473511953165, P41=0.5977615352548873, P42=0.5761318021233035, P43=0.5769756858888849, P44=0.5564301693135519, P45=0.5385961560205557, P46=0.5789605624434386, P47=0.6323154921086491, P48=0.7147677448288531, P49=0.6712814096286495, P50=0.6122610896461338, P51=0.6145260130085813, P52=1.691539104552654e-06, P53=2.1323343685410635e-06, P54=3.003829039387326e-06, P55=8.19376804143797e-06, P56=1.5532685112710195e-05, P57=1.572017427762086e-05, P58=7.369630894355265e-06, P59=3.448981210988018e-06, P60=5.125221548280725e-06, P61=2.291734358381622e-05, P62=0.00034242312772818503, P63=0.005777998158136341, P64=0.026020527779832464, P65=0.028495469664537677, P66=0.02019564660740061, P67=0.019463336732347954, P68=0.02600919239022271, P69=0.01595599738449052, P70=0.0045315475067811365, P71=0.0031978819580086005, P72=0.012606047897341764, P73=0.008315570941045609, P74=0.000772470479373543, P75=0.00017281098887013846, P76=0.001130096971385749, P77=0.005641125385451141, P78=0.009246629173275352, P79=0.0057910263192870815, P80=0.0022677558331074365, P81=0.0010066580247206935, P82=0.0014043017422662302, P83=0.001799262595751908, P84=0.03264045164773432, P85=0.10003390003050046, P86=0.027259765534826785, P87=0.0036814454359353757, P88=0.0009538051725187801, P89=0.0017674440581709502, P90=0.0023713111280640334, P91=0.003929027508209537, P92=0.0027073558793517484, P93=0.0003858484585123406, P94=0.003427127483141603, P95=0.004657907769109164, P96=0.0018910189865553567, P97=0.00162865080506376, P98=0.0022723023365611426, P99=0.0014287335627219543, P100=0.00021162700136505256, P101=0.0002049984791322937, P102=0.00017608319605784647, P103=0.001325022830951855, P104=0.0040822136626691334, P105=0.004096074189229512, P106=0.0027996651421166173, P107=0.00047261669240469367, P108=0.00024036425225555704, P109=0.0013882177540001969, P110=0.011979493349791107, P111=0.008407417169716774, P112=0.0035994007845789777, P113=0.0005381953310732297, P114=8.349787735658257e-05, P115=0.001525586882283073, P116=0.003773679408376362, P117=0.002144208346339547, P118=0.0012195371118366445, P119=0.0004912100349689207, P120=0.0006826654381259437, P121=0.0010456227429927098, P122=0.0004281033751013784, P123=0.00035858466565015724, P124=0.0003376930479050434, P125=0.0002050447336726094, P126=0.00022981544221890897, P127=0.0001883289793443966, P128=6.644682646774918e-05, P129=6.381117385449358e-05, P130=8.709034155339109e-05, P131=2.5397714236405338e-05, P132=5.086584504687193e-05, P133=4.1420600666077844e-05, P134=0.00013781852904904364, P135=0.00012840095024407438, P136=3.273249574280689e-05, P137=0.00017834781493097118, P138=5.6192823456803565e-05, P139=6.842775550986428e-05, P140=0.0001349416613814189, P141=0.00012816186247819017, P142=0.00017450312229145025, P143=0.0001206831879211741, P144=9.072191102721462e-05, P145=7.005004975598633e-05, P146=4.7517078519585886e-05, P147=4.6694944640672587e-05, P148=3.6205799067408083e-05, P149=3.0366863032582767e-05, P150=3.585566214855077e-05, P151=3.6643604105510503e-05, P152=2.002359972278201e-05, P153=2.3414150597662622e-05, P154=1.3035889443223976e-05, P155=1.2296656520721385e-05, P156=1.3008816147494649e-05, P157=1.4215297832053804e-05, P158=2.1748731138384566e-05, P159=2.0061354245876197e-05, P160=1.5133495809948721e-05, P161=2.0089518647824822e-05, P162=2.200679869025157e-05, P163=2.9415648457800862e-05, P164=1.64002442629021e-05, P165=1.245446673976102e-05, P166=1.0951244323535225e-05, P167=1.610515355166347e-05, P168=2.6615459075955437e-05, P169=4.244034441988567e-05, P170=3.4393920563922374e-05, P171=2.8942836250468485e-05, P172=2.791814736087573e-05, P173=1.7448964413632368e-05, P174=2.1309255057744577e-05, P175=1.4217351434222651e-05, P176=9.005520767295272e-06, P177=9.60193987798973e-06, P178=9.48798145005046e-06, P179=1.4545633039543503e-06),Row( P0=-406.683642615284, P1=31.536772909634184, P2=-30.713923638394295, P3=-9.467341236803014, P4=-21.11373626467812, P5=-19.68072293079503, P6=-22.668731784874918, P7=-7.109352040041786, P8=-0.9786542901258887, P9=-7.153333138196151, P10=-10.336315921657759, P11=-0.6824510229671007, P12=5.491610533973082, P13=4.100433224154032, P14=9.290701550248741, P15=12.200306932134731, P16=10.436068079572768, P17=4.37358671734885, P18=-6.047351591671645, P19=6.90772154149723, P20=-0.303848824211158, P21=2.3011196728313768, P22=4.050233739252998, P23=-0.8193715766274902, P24=-4.1886835692788615, P25=-1.1822097643306018, P26=-1.0721723180371692, P27=3.4208979255328225, P28=2.2379682284877274, P29=0.08423709983678077, P30=-1.6614146922999986, P31=2.6890574639080733, P32=-1.8941322703866599, P33=-0.515239013297392, P34=-4.688882466522721, P35=-0.7423354673800092, P36=0.8279177622848382, P37=-0.10112201070961427, P38=-2.1321267453409973, P39=-1.350892805111062, P40=0.5699852188720489, P41=0.6088744321446347, P42=0.6437065335352204, P43=0.6199070128928028, P44=0.5760308447468725, P45=0.6536599210390458, P46=0.686425911413032, P47=0.5988564227584966, P48=0.619979797296871, P49=0.6196601453884291, P50=0.5898502681830816, P51=0.5810486425880836, P52=0.004487785586149788, P53=0.004362203104004469, P54=0.0029922114588448903, P55=0.0011917759715772852, P56=0.000619464039185251, P57=0.00038732957678370153, P58=0.00027014257122445344, P59=0.0006426429361685509, P60=0.007670289800957038, P61=0.04322708115038045, P62=0.09513968227666617, P63=0.11320458357163903, P64=0.13977291407321163, P65=0.3986831500189562, P66=0.656884157626228, P67=0.6290781925399024, P68=1.4089533926576492, P69=4.108226187473447, P70=4.759777546478092, P71=2.6584086625250523, P72=0.8290529474278141, P73=0.6377585179488223, P74=0.3897233555164888, P75=0.19699424756268333, P76=0.12399010910339482, P77=0.18914780145387527, P78=0.2059508226757651, P79=0.3301548187050257, P80=0.39781724259043283, P81=0.30746455236871173, P82=0.43336175975590074, P83=0.6040090129902549, P84=1.0826654138582368, P85=1.5703710679031722, P86=3.243782099160292, P87=4.112082398823001, P88=3.3834176337360673, P89=2.9705930376307785, P90=1.8506924625447831, P91=0.6615135957058228, P92=0.6308652726650307, P93=0.28565619669970627, P94=0.2815442050777484, P95=0.24312468090339753, P96=0.1556168273588515, P97=0.31643283724351706, P98=1.1054317790712676, P99=0.9351713515824329, P100=0.48825726519981505, P101=0.25632048864340856, P102=0.4805900404053787, P103=0.29072487756046955, P104=0.2596456894479645, P105=0.11680723871555679, P106=0.12664905887128589, P107=0.3425891418200823, P108=2.4273322905292964, P109=1.768644810869271, P110=4.160371427666785, P111=3.8870686963733756, P112=0.0981296404643632, P113=0.023349187906132662, P114=0.2149935380739306, P115=0.32106505940060126, P116=0.2213966829942279, P117=0.07926039684588645, P118=0.06159988821355631, P119=0.05597168878400182, P120=0.26267039744721105, P121=0.2646963136013796, P122=0.16516506200666767, P123=0.10748164375109372, P124=0.047648626799482646, P125=0.019482410867125103, P126=0.010498700804508368, P127=0.012302995807075766, P128=0.04278715086177725, P129=0.03939060559435808, P130=0.014987801787002875, P131=0.02328371748650969, P132=0.09487865729257833, P133=0.0868112596897216, P134=0.06570904939611764, P135=0.08736445888635239, P136=0.03596618844823476, P137=0.009291913103127583, P138=0.009091447071054115, P139=0.01590971146655022, P140=0.020640493294541082, P141=0.02048877504541174, P142=0.03836820659507722, P143=0.04740950695667422, P144=0.07376646022253286, P145=0.0389047862070321, P146=0.01512474295955272, P147=0.009752703452136724, P148=0.002794115380231338, P149=0.004458875427950584, P150=0.0040343288241725785, P151=0.007151455486582896, P152=0.00819524553065271, P153=0.004017620733693327, P154=0.003509079921376846, P155=0.0037510717034382472, P156=0.003894927756911979, P157=0.0043603157154908975, P158=0.007036254327984632, P159=0.007082233915349787, P160=0.006536449514552707, P161=0.0041155648964246545, P162=0.0023150818344493847, P163=0.0013568662125731455, P164=0.000961089336287665, P165=0.0008956328452192469, P166=0.0014187229450416102, P167=0.0026829964813958416, P168=0.0018500034455910886, P169=0.001526903278311567, P170=0.0013948292898183518, P171=0.002295445145237371, P172=0.003389149792908794, P173=0.0034903306232680936, P174=0.0036685965407348715, P175=0.0025982542521748504, P176=0.004409682351825567, P177=0.004968749867667409, P178=0.00239438296336623, P179=0.00025798542761524857),Row( P0=-647.5773391068707, P1=51.371345845529106, P2=-12.269239956175035, P3=-5.211579811213721, P4=-12.799995330421725, P5=-16.117927247310025, P6=-12.453923706435631, P7=-2.8025034382021956, P8=-5.596651175262216, P9=-2.800035399254521, P10=-10.327944269528412, P11=-3.079456680909826, P12=-1.2631275735756071, P13=-6.750883901858671, P14=-3.2820147263856105, P15=-3.550820185678868, P16=-1.626001373438081, P17=-2.168579703677688, P18=-0.2618040115725655, P19=1.142023637284407, P20=-2.661613431022685, P21=-0.6223345796646444, P22=1.3120767386211758, P23=2.9603274716249737, P24=0.000258952626923564, P25=3.8847143178030947, P26=-1.7151103197973978, P27=3.466246991386126, P28=-0.7124087605010702, P29=0.6062006710124503, P30=0.925986018339714, P31=2.189660039770817, P32=-1.3938563384345692, P33=1.6792692238183329, P34=-1.9086738657087652, P35=-0.773987829904562, P36=-3.6096610559334983, P37=0.24476363944201263, P38=-0.6925635270312699, P39=1.8032462120327233, P40=0.6906575433242, P41=0.6465038141613607, P42=0.6396007817409155, P43=0.6982542267707668, P44=0.7189459778190802, P45=0.7175811510456347, P46=0.6818763454609604, P47=0.6701243489762069, P48=0.7122801310519564, P49=0.6798339088324105, P50=0.6246069054041942, P51=0.6373159696972374, P52=1.074613044621114e-06, P53=6.0321544435330925e-06, P54=8.305137030013017e-06, P55=6.836899950326626e-06, P56=2.616548195063822e-05, P57=0.00015611026749729642, P58=0.000374840970981031, P59=0.0037761924761487947, P60=0.013605958143536896, P61=0.02332851974267948, P62=0.033640919858080695, P63=0.025115140275150837, P64=0.01817919323888897, P65=0.02324940822678451, P66=0.036989465766084736, P67=0.06213456166938301, P68=0.028244053941472943, P69=0.004693083898740694, P70=0.003601009087612083, P71=0.003959142352539631, P72=0.005760963697147002, P73=0.008542664574008787, P74=0.006460704393125021, P75=0.00451680434221115, P76=0.0067348111926179306, P77=0.020658757526321903, P78=0.031902333295710544, P79=0.02298121374531559, P80=0.016199357021681137, P81=0.016680129676703397, P82=0.01887111259762909, P83=0.049357711127276854, P84=0.04963314502876338, P85=0.01391507181090162, P86=0.003087764235013373, P87=0.004170508032960393, P88=0.0046264139320582294, P89=0.006858112186840732, P90=0.010447047175477443, P91=0.007632337306879571, P92=0.0023772063520028242, P93=0.0010217350867235786, P94=0.0012482885690155694, P95=0.0010585925909209553, P96=0.0010065119999890816, P97=0.0008169865290299423, P98=0.001172532844517764, P99=0.0012362988618056876, P100=0.0007668190251733668, P101=0.00023504580544688025, P102=0.0003428015955480297, P103=0.0005494870426230249, P104=0.0006885169119136179, P105=0.0007487009160616419, P106=0.0005859645759494571, P107=0.00019265177040257578, P108=0.00019460414175673392, P109=0.0003127970005098199, P110=0.0007610006752509308, P111=0.0006816666049986904, P112=0.0001595915553251836, P113=0.00021478261364440507, P114=0.00031559468235628827, P115=0.0003787198568919966, P116=0.0004119042050385629, P117=0.0004913361924109394, P118=0.0003787328655784583, P119=0.0003369915823246533, P120=0.0003154451934220896, P121=0.0002669477595455154, P122=0.0002759564521818621, P123=0.00012395864771616184, P124=0.0001250484160994091, P125=9.980537169048596e-05, P126=0.00015926137152414696, P127=6.944925845482811e-05, P128=6.348735660320261e-05, P129=0.00011695924837776015, P130=5.084912680875467e-05, P131=4.4138536563042915e-05, P132=7.050507406959669e-05, P133=4.879483010063879e-05, P134=9.322219001341174e-05, P135=0.0001480012989044955, P136=0.00015784833222587844, P137=8.129488227606148e-05, P138=5.786985529286517e-05, P139=3.125443053664406e-05, P140=2.5147700544553423e-05, P141=2.6830277292082354e-05, P142=3.5581681214327376e-05, P143=4.5535419802114814e-05, P144=7.206423388630783e-05, P145=0.00010235318001639411, P146=0.00011472339370697524, P147=5.9309695440146365e-05, P148=4.081335766599933e-05, P149=2.291105815920568e-05, P150=2.2469135843700632e-05, P151=3.158999976073908e-05, P152=8.999340162552577e-05, P153=7.822935331143578e-05, P154=0.00010497344593617312, P155=8.027498384083509e-05, P156=0.00013169605475411736, P157=9.840304894225848e-05, P158=5.1131681662843625e-05, P159=3.600205341139087e-05, P160=4.003172333904074e-05, P161=6.316570219910116e-05, P162=7.244107932941254e-05, P163=7.587270832459691e-05, P164=6.658161236010646e-05, P165=7.096354157542598e-05, P166=3.944489191993418e-05, P167=3.915026309125686e-05, P168=2.8052471767390292e-05, P169=1.904642471748816e-05, P170=1.9447420663480607e-05, P171=1.5098107263601772e-05, P172=1.0248365327682252e-05, P173=8.169171110162127e-06, P174=1.1150893155720151e-05, P175=1.46284965646883e-05, P176=1.927638143152355e-05, P177=1.1352003452538447e-05, P178=4.619161538550061e-06, P179=4.67243141439641e-07),Row( P0=-539.4101607157595, P1=41.83126588383007, P2=-21.460953691469644, P3=-2.9602399977469482, P4=-19.28115961910091, P5=-17.291326608901983, P6=-13.732418586333843, P7=-3.52339967009588, P8=-4.998644875984167, P9=-3.94223001079042, P10=-16.239123525723716, P11=-6.462982256256586, P12=-4.876840314005399, P13=-11.798267082959638, P14=-3.220795740256857, P15=-8.140944541439104, P16=-3.4940133972176266, P17=1.1122725375705549, P18=-2.628826822343183, P19=8.739266668436533, P20=0.7498543377989338, P21=4.309181687950833, P22=4.432880153658603, P23=0.8753731877376284, P24=-0.5083839113739721, P25=2.600062314191221, P26=-1.7636404893100703, P27=-0.3051014984920111, P28=-0.5663852693999064, P29=-1.8142965449649302, P30=-1.7834889520277777, P31=-0.05007003302401956, P32=-3.2533289680447925, P33=-0.8554425079263915, P34=-4.277891054974852, P35=-0.8565191362939352, P36=-1.3938337590438108, P37=-1.8508786818794383, P38=-1.7589916705420712, P39=-2.969526795742445, P40=0.6426311741376156, P41=0.6173154013786822, P42=0.6377630855993686, P43=0.6270957936681726, P44=0.653173739276306, P45=0.6861531690797062, P46=0.7206696504579173, P47=0.6962482363754731, P48=0.7006756400378807, P49=0.7188886654346719, P50=0.6808953129383748, P51=0.6562526559879045, P52=5.44329851091467e-06, P53=4.941769098634757e-06, P54=2.2617851561264494e-05, P55=4.09047537574815e-05, P56=9.87887042739869e-05, P57=0.00042241018954592856, P58=0.000920275973564759, P59=0.0037447588948136524, P60=0.014692415077284722, P61=0.053399516256736355, P62=0.16397423397674155, P63=0.27711564172170955, P64=0.20515476804297275, P65=0.28195320470748936, P66=0.15340351710628966, P67=0.0187402111018134, P68=0.0064400836666682094, P69=0.005470384925147733, P70=0.0087715837434186, P71=0.010786017142755669, P72=0.018813262629172014, P73=0.030965299985104153, P74=0.0751556749162604, P75=0.10058583143190487, P76=0.1396793785699399, P77=0.32352213255210865, P78=0.6549982309548202, P79=0.6185228971555822, P80=0.296201548505522, P81=0.08628442929429082, P82=0.05700469851233836, P83=0.06995730212884024, P84=0.11722050174241909, P85=0.2744231686443656, P86=0.19154149726360664, P87=0.19037376916011656, P88=0.27122390295968934, P89=0.1787395362344089, P90=0.07021571809804192, P91=0.042988257089493775, P92=0.16949610475847848, P93=0.10191799143763479, P94=0.06800836657773544, P95=0.07807690286095302, P96=0.13009146162916885, P97=0.0247300458652746, P98=0.006804856023948255, P99=0.003076587108146813, P100=0.0038159177824389763, P101=0.007842135772317451, P102=0.01399861976500141, P103=0.021357349010262427, P104=0.024803209966145084, P105=0.007616096549770345, P106=0.0031174438067643386, P107=0.002940823296033535, P108=0.014123350606167337, P109=0.01610141208008555, P110=0.011790509444891956, P111=0.00995066062195982, P112=0.0034718213152326893, P113=0.008468343463954787, P114=0.021059117118349444, P115=0.01013804473650132, P116=0.007763821062180228, P117=0.03183759801769334, P118=0.016575712092901703, P119=0.007815790355394992, P120=0.015333849931703338, P121=0.012836692157560335, P122=0.011472931017011976, P123=0.009902453450197643, P124=0.006111511109538201, P125=0.005621444972502254, P126=0.00591497834608904, P127=0.002287534385467953, P128=0.0015881781707806567, P129=0.002171824564325999, P130=0.002152632320477241, P131=0.0013434759608083064, P132=0.0077189353048633036, P133=0.0069995530757302514, P134=0.004115557184216211, P135=0.003491068587580008, P136=0.0011964491874799249, P137=0.0011215346322727459, P138=0.0009985428779615071, P139=0.0007622989096222612, P140=0.0014796548789179093, P141=0.0024944295614113786, P142=0.004776029150413972, P143=0.004056400407909845, P144=0.008733756796409697, P145=0.00987599834468459, P146=0.0031884555505356893, P147=0.0006911184788134442, P148=0.00013777242476559903, P149=8.396563961933478e-05, P150=7.65257006688538e-05, P151=9.583745492022098e-05, P152=0.0001482619128495677, P153=0.00016610732866808336, P154=0.0003018717383057265, P155=0.0005166822301352452, P156=0.0004373069930342316, P157=0.0003941663884795724, P158=0.0005527098384185674, P159=0.0008541821097903363, P160=0.0008625067141367264, P161=0.0007453301072709956, P162=0.0007634909342668309, P163=0.0005252920520470906, P164=0.00036449354177114717, P165=0.0002798429450415698, P166=0.00020935521409810718, P167=0.00024383972097790298, P168=0.0001388569923411472, P169=0.000139825543373996, P170=9.354205738253264e-05, P171=0.00010086943569053534, P172=0.00010999586309420275, P173=0.00011349693103995843, P174=7.34767213797023e-05, P175=5.271002177579639e-05, P176=4.004026222462061e-05, P177=3.693048753414138e-05, P178=2.2561229064956944e-05, P179=3.6973458897055074e-06),Row( P0=-695.2913880071713, P1=54.17679665488948, P2=-11.570376511526714, P3=0.5456251551545115, P4=-13.325181796524971, P5=-8.15654419152783, P6=-11.398822165634785, P7=-1.0520188700101816, P8=-1.3795533473398167, P9=-7.719797764173477, P10=-9.254503821269703, P11=-3.7673689761269302, P12=-4.150581703592993, P13=-7.469933966040864, P14=-4.644456631521902, P15=-6.477122923464713, P16=-1.0524037004842228, P17=-5.378818080920078, P18=-5.770055772608958, P19=-1.8118703846478101, P20=-7.158383366410666, P21=-3.8854921057620904, P22=-1.2864114361457206, P23=-0.5875583230037574, P24=-1.9211184349241983, P25=2.8006560801994085, P26=-0.06519162455580838, P27=2.043432930809753, P28=2.657587878576425, P29=3.216397337855424, P30=0.8360187249287883, P31=2.5839518594037023, P32=-1.291292238180688, P33=1.1850218643417072, P34=-0.9169919238651403, P35=-0.5935390549460127, P36=-0.6620022961652782, P37=1.2804519952873514, P38=0.10018140185977467, P39=1.3599017368327555, P40=0.6806791828600253, P41=0.6978482174317631, P42=0.7291281458841702, P43=0.7217007852236136, P44=0.7331996601957579, P45=0.739524608055447, P46=0.7100365153433603, P47=0.7002287944911882, P48=0.706666302924687, P49=0.7139702774817496, P50=0.6873564177527016, P51=0.6989433435692218, P52=6.865520136008303e-07, P53=4.1036031413518925e-06, P54=5.140692643576631e-06, P55=3.854159102017328e-06, P56=0.000254852434745881, P57=0.0025073008168555104, P58=0.004577150365644978, P59=0.010209977276118961, P60=0.028186572779302494, P61=0.040726849673372124, P62=0.030693968905673524, P63=0.017802454391884614, P64=0.00697067486741997, P65=0.0027308224890963875, P66=0.004713266445425387, P67=0.005065435188457745, P68=0.010996937037606564, P69=0.02498519291333118, P70=0.020463253065660506, P71=0.02194835433377748, P72=0.016309995144506477, P73=0.008742009277818649, P74=0.006408335483183634, P75=0.007204767496829843, P76=0.006089699007088608, P77=0.005040131299565475, P78=0.008253069432610427, P79=0.007238426315807911, P80=0.007016583781680103, P81=0.010125988909576555, P82=0.01129376918350669, P83=0.00769784351178043, P84=0.011835565671048213, P85=0.011079397649788533, P86=0.021537900082330058, P87=0.03685734517716686, P88=0.019587164784367346, P89=0.003335838867124541, P90=0.0025173065074565568, P91=0.0033147327449364068, P92=0.00334744177042559, P93=0.004519456201846865, P94=0.006792663946440813, P95=0.005316597381944469, P96=0.005021809456875746, P97=0.0026812879234464878, P98=0.0014459137282222488, P99=0.0017478489017435114, P100=0.0007306342526842836, P101=0.0003368113079608389, P102=0.00034613204112570167, P103=0.0009517387974559243, P104=0.000580265727639198, P105=0.00042459857037912733, P106=0.0006635934585468755, P107=0.00034612841676122125, P108=0.0005440467688796127, P109=0.0009527331412478699, P110=0.0010742605209940563, P111=0.0014378308847781403, P112=0.0005061696225434543, P113=0.0010099831887062222, P114=0.0027061315508020586, P115=0.0016834182316383425, P116=0.00041483904948100406, P117=0.00027061991511054, P118=0.00025929645503904166, P119=0.0002171826353796924, P120=0.00055831655705752, P121=0.0006318630257344095, P122=0.00025202571496672484, P123=7.972779070844869e-05, P124=5.278132419259706e-05, P125=9.28882221379158e-05, P126=3.4365144831580174e-05, P127=3.869826589588746e-05, P128=3.686141596046644e-05, P129=5.4923389672233035e-05, P130=4.69177411374677e-05, P131=7.681762199268688e-05, P132=0.00010196049861279104, P133=9.093115604847522e-05, P134=5.528202606663307e-05, P135=6.436684737128268e-05, P136=2.901238113852752e-05, P137=1.2894155046614593e-05, P138=7.741128754164591e-06, P139=8.432192482422023e-06, P140=7.309455629989086e-06, P141=1.1552099688761768e-05, P142=2.8364798624828278e-05, P143=6.379334498784557e-05, P144=7.468600637354625e-05, P145=9.37227251350129e-05, P146=9.207479252444554e-05, P147=3.1910856671693386e-05, P148=2.615805481661515e-05, P149=2.9643946113688445e-05, P150=4.18769512337073e-05, P151=3.4203129187916174e-05, P152=3.42209040695891e-05, P153=4.186819645239373e-05, P154=8.212869024284847e-05, P155=0.00016917433969782173, P156=0.00012963626586039943, P157=0.0001286511538461469, P158=0.0001512912427566737, P159=0.0002359484727825971, P160=0.00017196547864354908, P161=0.00010036857301960663, P162=7.141967794884836e-05, P163=6.152570635122497e-05, P164=5.227580972917134e-05, P165=6.896074116132127e-05, P166=6.48277983759094e-05, P167=3.90630449174711e-05, P168=2.3503846393453058e-05, P169=9.50676807215868e-06, P170=8.503050273919435e-06, P171=9.932935992596896e-06, P172=1.1029850039198697e-05, P173=1.1672457192557773e-05, P174=1.5114069586976336e-05, P175=1.9013453343086018e-05, P176=1.3758503100139334e-05, P177=6.4314308338080824e-06, P178=2.991415612790485e-06, P179=4.868938287774219e-07),Row( P0=-622.3715315095798, P1=45.516600641482306, P2=-18.506124355916818, P3=-5.483932246232914, P4=-17.92410895578107, P5=-15.728436828506585, P6=-14.360893463815435, P7=-2.7777925540760937, P8=-7.312738773018974, P9=-6.578647448590021, P10=-10.904776190311685, P11=-9.531343764874897, P12=-3.247154416624932, P13=-7.893803993867353, P14=-1.6552070208369594, P15=-2.8307853904427582, P16=0.1924594970462593, P17=3.449351243317218, P18=-1.3319491068409475, P19=3.9014414639106825, P20=-1.5035652059185747, P21=2.6033180214574156, P22=2.613546660998463, P23=-0.41641948988699234, P24=-2.1720971081124474, P25=-2.189015795280314, P26=-4.290382269908877, P27=-1.4173083087451877, P28=0.481124220126129, P29=-0.05777637040245295, P30=-1.6314189155264953, P31=-1.469740251842385, P32=-2.429492379558664, P33=1.3096434063854991, P34=-1.846099525593804, P35=-3.349486691873049, P36=-5.22482994186557, P37=-1.4017657907435093, P38=-0.2847184255018194, P39=0.280949808018133, P40=0.7044221640082622, P41=0.6897509177605243, P42=0.6387830532004259, P43=0.6727936256720569, P44=0.7259064274496114, P45=0.6741996338722883, P46=0.690001898937224, P47=0.7060384522194594, P48=0.7192038960165018, P49=0.713555929368211, P50=0.6413405679209361, P51=0.6256855879901353, P52=2.3430994561529699e-07, P53=1.2671325555293413e-06, P54=1.1022929402976118e-05, P55=2.7877329162478925e-05, P56=4.1388836971122547e-05, P57=0.0001231362592608436, P58=0.00020690227549169932, P59=0.0004923601503772401, P60=0.0021749925404578956, P61=0.00913370397502697, P62=0.025358560882038205, P63=0.0500036892965289, P64=0.060437886341610104, P65=0.06593850144054884, P66=0.0790627923558828, P67=0.09285840937452003, P68=0.1185388551900448, P69=0.10009554828252115, P70=0.031129165072475644, P71=0.007963569176004744, P72=0.005480607384284072, P73=0.008714034592516458, P74=0.008542266886606845, P75=0.00848319351515802, P76=0.020709105904797895, P77=0.08793941126404183, P78=0.11114489992682529, P79=0.0630228916293829, P80=0.025472150544360743, P81=0.017412506846654196, P82=0.020426333479463512, P83=0.037417019281939135, P84=0.0818520069698642, P85=0.08502115357929452, P86=0.09269521132901293, P87=0.06337708617831356, P88=0.024371046935890543, P89=0.07924154581886031, P90=0.13765233969391408, P91=0.053378296468340046, P92=0.01915304158567162, P93=0.00830767987667986, P94=0.0213337474146242, P95=0.030694582001067748, P96=0.07023403112090655, P97=0.11070312664439141, P98=0.03026474374778816, P99=0.01731033933457413, P100=0.002988746454356812, P101=0.002033578917367838, P102=0.004722763929709548, P103=0.007554620259027266, P104=0.011854026738266776, P105=0.02421184687527475, P106=0.021628858369943427, P107=0.003115947979131055, P108=0.001096535530606986, P109=0.0015782117897542014, P110=0.013929940084818725, P111=0.038656301597124854, P112=0.047768643601561314, P113=0.02043388129150982, P114=0.002855765032086111, P115=0.003471806343070469, P116=0.007388184547647938, P117=0.0034541027390460766, P118=0.0010732778738905744, P119=0.0009797837557435508, P120=0.006189890069336867, P121=0.0045452978742668345, P122=0.000997865674840059, P123=0.0017407672811523367, P124=0.0009930270112270478, P125=0.0004940863110210101, P126=0.0006805545456136709, P127=0.0009323063097736842, P128=0.0011421272439562503, P129=0.0007099241668621682, P130=0.000653355705279675, P131=0.0004408368337077512, P132=0.0009111845059617457, P133=0.0017875221476532294, P134=0.0007159544011643138, P135=0.0007145279464046694, P136=0.0005089165529722706, P137=0.0003100440264074312, P138=0.00019280810005645597, P139=0.0002259735809850036, P140=0.00015275633249692104, P141=0.00016560911618312857, P142=0.00026421108380781977, P143=0.000169333231543099, P144=0.00019114540187326106, P145=0.00029121078201612473, P146=0.0004927348744523189, P147=0.00033132375928301126, P148=0.0002192363289499261, P149=7.410881395215373e-05, P150=7.937971597521632e-05, P151=9.197884398719467e-05, P152=0.00010663755965117962, P153=0.00019913132295437195, P154=0.00017086538536928404, P155=0.00017480810760858126, P156=0.0001761769183953396, P157=0.00017569727927498884, P158=0.00015633628041988951, P159=0.00013079095820611573, P160=8.875345613225978e-05, P161=7.692993837030939e-05, P162=9.831773745348306e-05, P163=6.53958965640649e-05, P164=5.362658704373843e-05, P165=5.672138178120635e-05, P166=3.54769891591051e-05, P167=3.591245263156001e-05, P168=3.3944056730065936e-05, P169=3.7098363591562026e-05, P170=2.6315406377707324e-05, P171=1.176271225158706e-05, P172=2.109179932622015e-05, P173=2.39989968170054e-05, P174=1.9581459940734713e-05, P175=1.2931619590501394e-05, P176=1.1126037068099373e-05, P177=1.5129758406950309e-05, P178=6.5808488174462814e-06, P179=1.0267861031935226e-06),Row( P0=-741.8659484721783, P1=64.79643443491568, P2=-4.849465763155923, P3=5.95499649833459, P4=-10.49910460741163, P5=-11.896031438423236, P6=-7.430050833737218, P7=-1.627411086569049, P8=-1.5251628136659747, P9=-0.7494457006388129, P10=-8.878161381568544, P11=-6.106435654687338, P12=-2.9270567519297894, P13=-6.966548007447807, P14=-3.6811999198298926, P15=-5.08046996161742, P16=-1.0577715799573593, P17=-3.3227037907587307, P18=-4.341139426593781, P19=-0.5391969027346728, P20=-7.531218807451208, P21=-4.219599006742952, P22=-4.171323169861643, P23=-3.9365903084836225, P24=-5.029708086190342, P25=-1.2237694744864753, P26=-3.6703937846081685, P27=-2.1194913055831583, P28=-1.0758731607786165, P29=-3.1770705870070377, P30=-1.9744572460800207, P31=-0.2637812191084635, P32=-2.9940013208126586, P33=2.676120229190264, P34=-1.3672895663482179, P35=1.2901089388350024, P36=-2.117920939650146, P37=-0.05097701558503935, P38=-0.4111622646248328, P39=0.7824060152746799, P40=0.7550632012768209, P41=0.6805373709479713, P42=0.7218655003658248, P43=0.7638359608434667, P44=0.7366865583353578, P45=0.7053546711859157, P46=0.6971582347596156, P47=0.6798501316363531, P48=0.7154411592355746, P49=0.7292657650779589, P50=0.7424502004173618, P51=0.7795229216404042, P52=2.4650596317721837e-07, P53=8.754826247279734e-07, P54=8.658935032987207e-06, P55=2.2807566897140545e-05, P56=9.794125712965827e-05, P57=0.0013768325356305706, P58=0.004983155923701375, P59=0.009218181582071652, P60=0.009382191462919158, P61=0.014635020498022757, P62=0.010050926885048316, P63=0.0009681247081285519, P64=0.0008620938722301616, P65=0.0033000810215490393, P66=0.006452240757208838, P67=0.009327292325601822, P68=0.007392453767714787, P69=0.005888409062139485, P70=0.0061354324249600295, P71=0.007085743706484047, P72=0.011690008654136275, P73=0.005820240989256822, P74=0.0015432517034508785, P75=0.0017364133190190126, P76=0.0011478682091728167, P77=0.0015671330631159186, P78=0.0031281295241230747, P79=0.004560811690226973, P80=0.0057688641855941055, P81=0.006213760934918622, P82=0.011097068063184019, P83=0.011489713489582, P84=0.003603016874195034, P85=0.0011085182207653558, P86=0.0007147425584459828, P87=0.0007599599247869668, P88=0.000907526206962657, P89=0.004269872538281594, P90=0.0062069846034457235, P91=0.0010296190535494668, P92=0.00034808044057897815, P93=0.0001299046622481919, P94=0.0004239302465542488, P95=0.0020927908389783787, P96=0.0022493592597005803, P97=0.0004904925630343129, P98=0.00016001837597210238, P99=9.492719139313179e-05, P100=0.00016797828090090263, P101=7.48176791847162e-05, P102=4.354162289397874e-05, P103=3.458066616664363e-05, P104=9.486837437466697e-05, P105=7.81555027739387e-05, P106=0.00017825376841020378, P107=5.508763602232239e-05, P108=7.266492154832003e-05, P109=0.00010516863792949858, P110=0.0005499335534574345, P111=0.00048789499271983104, P112=7.695012270721975e-05, P113=8.820974957959163e-05, P114=0.00036566414558481604, P115=6.775538448645701e-05, P116=0.00010730597028274569, P117=0.00017332495010543157, P118=4.346438352041542e-05, P119=0.0003678167434029816, P120=0.00021715833906853005, P121=7.359944384701802e-05, P122=0.00010480533789980439, P123=9.696313701792836e-05, P124=9.101164289791374e-05, P125=0.0003714683558504927, P126=0.00011188914848246384, P127=3.596803217673515e-05, P128=2.8109575926184718e-05, P129=2.945464450261068e-05, P130=3.8822404779473315e-05, P131=4.358133616036425e-05, P132=6.851646225810806e-05, P133=6.25828124754271e-05, P134=0.0001579603783911334, P135=8.227563931969212e-05, P136=6.352615314199523e-05, P137=2.40033913455685e-05, P138=1.730815847054568e-05, P139=6.244142666007661e-06, P140=5.52274799990484e-06, P141=7.1081165225218834e-06, P142=1.2096512881685232e-05, P143=1.6618862384661416e-05, P144=2.519866271201623e-05, P145=3.687961981781776e-05, P146=3.309930403543852e-05, P147=1.775701017477027e-05, P148=4.242447279793607e-06, P149=2.6882825324684256e-06, P150=5.097397612433066e-06, P151=7.281336009314715e-06, P152=1.0546008194986992e-05, P153=8.606488459037327e-06, P154=1.862621411182428e-05, P155=1.5816483615053835e-05, P156=1.0623426776587885e-05, P157=1.3758787524160046e-05, P158=1.6086532684546448e-05, P159=2.3493014511809323e-05, P160=2.1885180671156666e-05, P161=1.7738833840154685e-05, P162=1.4753530574895799e-05, P163=5.545885938230982e-06, P164=3.0139141306101414e-06, P165=2.5884808191404803e-06, P166=2.751233363428093e-06, P167=2.7984905747230746e-06, P168=2.4340595884273917e-06, P169=3.1023829151951175e-06, P170=1.8385690742516521e-06, P171=1.3146083630154694e-06, P172=2.0313026521562245e-06, P173=1.552330893935774e-06, P174=2.9548990073034197e-06, P175=2.406060690101462e-06, P176=1.263225924539305e-06, P177=1.5643307981796389e-06, P178=8.718712458367133e-07, P179=1.1934094256909818e-07),Row( P0=-721.0808137420591, P1=69.44343900315253, P2=-7.552645181292385, P3=2.3693660489668553, P4=-8.609372937379808, P5=-8.180166076293178, P6=-8.963900923453428, P7=2.201050477977801, P8=-0.6447970300349422, P9=-2.430505537911798, P10=-9.513499226487706, P11=-7.663382474380155, P12=-2.265682987779446, P13=-4.621327248580886, P14=-5.831258200561479, P15=-5.5886423028656145, P16=-1.8959268485897045, P17=-7.615645567396538, P18=-5.383018752651932, P19=-4.489231098944509, P20=-7.979924227399474, P21=-5.667205243646323, P22=-4.873079119544648, P23=-4.750902173478476, P24=-9.51917843023676, P25=-3.046907677257021, P26=-8.238991067289195, P27=-7.039478061956755, P28=-3.324718828536476, P29=-4.068418273009143, P30=-2.040360196761833, P31=4.191940405450049, P32=1.981302248836865, P33=8.56510064733795, P34=2.9415226755041184, P35=4.971481300986239, P36=-0.9175532677088378, P37=0.8473512185349492, P38=-1.0610444374023542, P39=0.19422062079642954, P40=0.7836433833489328, P41=0.6548069488891393, P42=0.6793611579129035, P43=0.7065904175276528, P44=0.6730654111936663, P45=0.6569881272733193, P46=0.6494100640112722, P47=0.6199404612436709, P48=0.6375634686839288, P49=0.6412325004201961, P50=0.6824632500151475, P51=0.7785407883837441, P52=1.0779054509136655e-07, P53=9.447907086146883e-07, P54=9.034877823490606e-06, P55=3.24593045849458e-05, P56=0.00025170953560904537, P57=0.0021817966063593895, P58=0.013586152962064349, P59=0.033256485242131824, P60=0.012353090625559981, P61=0.001229240157503197, P62=0.0003064970420324971, P63=0.00014411343137614481, P64=0.00023910307225274874, P65=0.0019463964797656105, P66=0.010355713769969584, P67=0.020366448107906058, P68=0.01162638343509231, P69=0.002596365426922748, P70=0.0008010198060440743, P71=0.00033640073834940326, P72=0.00048336970951736, P73=0.0009666504468154957, P74=0.002981183174763595, P75=0.004583115938415226, P76=0.003979224944615909, P77=0.003778812408395508, P78=0.0021940781112623564, P79=0.0016970993080038872, P80=0.001346728439839714, P81=0.001809164744760595, P82=0.002818955078515229, P83=0.0045383441058646935, P84=0.0045827222493478, P85=0.0018692634271362974, P86=0.0011024649517007174, P87=0.0020908081301667164, P88=0.007170912286048213, P89=0.00563961114800138, P90=0.0048530539880301945, P91=0.001903125264501896, P92=0.000364731874653798, P93=0.0009508442607240236, P94=0.001956134162447086, P95=0.0009713885494309199, P96=0.00024137963515698545, P97=0.00017257929688449546, P98=0.0005379756883209915, P99=0.00042433746705947405, P100=0.00030350552925767054, P101=0.00010600751905737651, P102=6.257406062340446e-05, P103=6.912267952436995e-05, P104=0.00012570728253863436, P105=0.00013846457241427444, P106=0.00019430837322047812, P107=9.221117490549188e-05, P108=0.00017140317091338846, P109=0.00022152724996904176, P110=0.0005462394559971202, P111=0.0006140617558445755, P112=0.0005199446682470898, P113=0.00016220914898599077, P114=0.00016613422931199305, P115=0.0003684261188169471, P116=0.00016905189850273305, P117=0.0001190076822017273, P118=0.00021913849260616673, P119=0.00012549561701579828, P120=6.035344723163776e-05, P121=8.334643701028451e-05, P122=0.00017702038357975453, P123=0.00012586476036629925, P124=2.0018321319441785e-05, P125=1.8309143277307926e-05, P126=1.4189873149573093e-05, P127=7.93464658540675e-06, P128=7.473006490712155e-06, P129=1.308209394109609e-05, P130=1.1000196988198546e-05, P131=1.9287457643807453e-05, P132=3.145683917231482e-05, P133=3.785013652242205e-05, P134=4.076332256591611e-05, P135=3.352502918869845e-05, P136=1.5941574001061595e-05, P137=4.800596012551099e-06, P138=4.827353736928947e-06, P139=3.207294009828907e-06, P140=3.3144008806191635e-06, P141=5.1959987782772495e-06, P142=8.874791591230944e-06, P143=1.2029457387121465e-05, P144=1.7971752986450605e-05, P145=3.072131973469771e-05, P146=6.687646433255395e-05, P147=0.00011490729062480711, P148=5.045034278126895e-05, P149=6.939282334608835e-05, P150=0.00013714359735687585, P151=0.0002787565989495115, P152=0.0005944827242091898, P153=0.0005313710235285969, P154=0.0004114617877281128, P155=0.00019327769437608173, P156=0.00012933329869339917, P157=8.109876983513925e-05, P158=7.891966776761863e-05, P159=7.706828092326496e-05, P160=6.582073448106764e-05, P161=4.520837633279094e-05, P162=3.1082422812443326e-05, P163=1.592591455671421e-05, P164=7.944306264927724e-06, P165=9.018561618734683e-06, P166=1.2900713268035813e-05, P167=1.5205948388563626e-05, P168=1.2546766403521243e-05, P169=1.1201030809018241e-05, P170=6.932232155063715e-06, P171=8.308692096970345e-06, P172=7.3690860416702055e-06, P173=4.495355521968264e-06, P174=4.007415383535796e-06, P175=5.8358563447059905e-06, P176=1.0554935855326327e-05, P177=1.2240221996013887e-05, P178=6.40544262525119e-06, P179=6.765794849490279e-07),Row( P0=-587.1593271663639, P1=50.02951570119489, P2=-23.331090761028705, P3=-1.5284296045235646, P4=-15.05477920578861, P5=-17.038556603243514, P6=-11.541804878127794, P7=-1.295755479760222, P8=-5.4626272098841975, P9=-4.173854899322366, P10=-14.156107954306984, P11=-9.020499560278664, P12=-1.2581212795254422, P13=-9.935353942707499, P14=-0.46273917383940866, P15=-1.8602626738382193, P16=-0.9685375611069479, P17=2.953078479647622, P18=-1.14401893600514, P19=7.473545707652413, P20=2.933696067318795, P21=3.668200520760791, P22=5.094737460329349, P23=0.004192886020093382, P24=-3.1984204645631, P25=-2.198149664182182, P26=-2.7675067950724035, P27=-1.2821882850143378, P28=0.47618430539245754, P29=-1.6719173474029407, P30=-2.5123371025574888, P31=0.215785645879249, P32=-4.237935036026882, P33=0.6778372948199953, P34=-1.4559944949161279, P35=-1.800620210336105, P36=-0.21799370987934222, P37=-0.73655019807471, P38=-2.4218237987725395, P39=-0.030645707178221446, P40=0.6375944437846659, P41=0.5894206530330559, P42=0.6448093063649779, P43=0.6827713456895124, P44=0.6260089081224381, P45=0.6446502706400555, P46=0.6928245700862582, P47=0.7104981513686242, P48=0.7088525891847249, P49=0.7464334808726562, P50=0.7194771043833642, P51=0.6517415193603521, P52=0.0003030581560468667, P53=0.00103127832601357, P54=0.0010576039501009216, P55=0.00023926290694552054, P56=0.00013125038988760453, P57=0.0003875942271930281, P58=0.0005381080680415121, P59=0.0008768255129741648, P60=0.0018062435452536002, P61=0.007241158349587388, P62=0.04126657776091369, P63=0.09445250821796311, P64=0.10529069519406266, P65=0.1533617155693639, P66=0.15648772764056013, P67=0.07568742633714753, P68=0.0857338561327105, P69=0.22546067998572167, P70=0.1327408499332306, P71=0.012911383538286537, P72=0.00454182649160942, P73=0.00642732399786505, P74=0.013093510186862384, P75=0.03303717452991574, P76=0.08863353591062673, P77=0.12876090574866397, P78=0.09096951217517649, P79=0.09438660784249156, P80=0.09567107380678741, P81=0.05669147902913124, P82=0.0472983986602158, P83=0.03496475198580806, P84=0.02347315256160775, P85=0.02218408274649682, P86=0.044507578121678326, P87=0.14339119553618518, P88=0.2765643470985844, P89=0.11778663469187844, P90=0.045978333486839704, P91=0.060099112999795604, P92=0.10965106480510506, P93=0.06981905453843491, P94=0.13586815760980372, P95=0.22768297687173825, P96=0.0332189904880126, P97=0.008142118205782296, P98=0.004902352067566426, P99=0.006955630680755483, P100=0.015308362352938417, P101=0.010223787077828813, P102=0.005488818107586567, P103=0.008978491411620979, P104=0.012025695942058035, P105=0.004324790550301839, P106=0.007306335867554931, P107=0.007346223476347251, P108=0.026588675695829674, P109=0.038528314285940224, P110=0.1331596906344705, P111=0.17786075034106755, P112=0.007447508769390343, P113=0.00885119847532812, P114=0.01682319644645566, P115=0.014700929792746372, P116=0.003521856364047591, P117=0.002157753922612552, P118=0.004381555671159434, P119=0.0037713333133013514, P120=0.002947856856135098, P121=0.0034746139033809647, P122=0.005209725053420692, P123=0.000990889265695949, P124=0.0013443791273138686, P125=0.00918123885331257, P126=0.009769466596722084, P127=0.0005630367427646153, P128=0.0011884986898061515, P129=0.0023955958123603404, P130=0.001217140128931396, P131=0.00106446414170201, P132=0.0010080689045077727, P133=0.0028408001771332105, P134=0.0020906532041583157, P135=0.0020218384884655126, P136=0.0018424607282875624, P137=0.0013924697034989582, P138=0.0004902177194014759, P139=0.00045075186546238437, P140=0.0003981070426587062, P141=0.0007083432328666482, P142=0.0010544352354312384, P143=0.0014327206381672094, P144=0.0010630235577506288, P145=0.002328942955236466, P146=0.00021588324207588686, P147=0.00018496436945531142, P148=6.262515142310472e-05, P149=3.1670539954219445e-05, P150=5.3225237952948035e-05, P151=3.556656755241778e-05, P152=5.252060374135737e-05, P153=8.752211673864025e-05, P154=0.00015036938391858765, P155=0.00011251490252307514, P156=8.009858911050498e-05, P157=0.000161152958423057, P158=0.00026632551468072483, P159=0.00028662080783375444, P160=0.00017063743166378147, P161=9.854735621156352e-05, P162=6.434813747432523e-05, P163=4.041386290336435e-05, P164=5.401225993184186e-05, P165=7.512800843999045e-05, P166=5.8032747577732916e-05, P167=7.992126325999914e-05, P168=3.8698217199146494e-05, P169=2.0428632214914424e-05, P170=1.5086665066464732e-05, P171=1.4111725050241674e-05, P172=1.7249163211158225e-05, P173=1.386502385671447e-05, P174=7.359996849812222e-06, P175=6.8913430252026705e-06, P176=1.0972103411454343e-05, P177=1.7002239481472546e-05, P178=8.105751940308953e-06, P179=1.0958856635411284e-06),Row( P0=-674.873803897461, P1=61.580292844950506, P2=-12.951543462411967, P3=1.5805436804726591, P4=-15.524003267071938, P5=-11.501955249668704, P6=-8.875988725769634, P7=-1.31943932271403, P8=-0.45721676878193157, P9=-1.613970717664249, P10=-12.05684981735193, P11=-4.515225141604025, P12=-2.047046822359478, P13=-9.173680801245348, P14=-0.5933783195228374, P15=-5.8275507093807875, P16=-1.1010538361498141, P17=-4.865580250647366, P18=-6.162039205389115, P19=-1.2424532728525042, P20=-8.285730716218694, P21=-3.9854448877061577, P22=-1.5943620569164716, P23=-2.579341311519041, P24=-1.6376424475830949, P25=2.108512261857424, P26=0.7294452788047832, P27=2.080619288790265, P28=3.8580174455519516, P29=2.5205247033729825, P30=1.7338036983275793, P31=3.1454775618528865, P32=-1.5590441279020257, P33=3.4481413346950407, P34=-0.7512717059979511, P35=0.426227357593497, P36=-2.4040154683382022, P37=-1.6905138609061268, P38=-1.7253823601457896, P39=-0.7007273068525016, P40=0.7409223960413812, P41=0.6984592346420231, P42=0.7273951083682343, P43=0.7489418006371379, P44=0.7104230568114449, P45=0.6540773921040212, P46=0.6461513727446007, P47=0.6479278102376327, P48=0.7014748390713429, P49=0.7147721493918795, P50=0.7142794247730424, P51=0.752330246258733, P52=1.9081974173601436e-06, P53=7.952351326560618e-06, P54=1.3448375133959932e-05, P55=1.9814893293462157e-05, P56=0.00011425020364606428, P57=0.0011066477471394192, P58=0.005439993349898391, P59=0.018735085980546493, P60=0.035345918690892174, P61=0.0685452230814221, P62=0.04624468674100499, P63=0.005911849139480933, P64=0.0009699438764757329, P65=0.0018605631160473924, P66=0.007663208042095213, P67=0.017319860041548577, P68=0.01964719645412747, P69=0.03189884895739356, P70=0.039071339330580344, P71=0.04438762476199317, P72=0.05346138106635626, P73=0.032693168473020395, P74=0.009758032339974387, P75=0.010812155954905651, P76=0.008159437208335372, P77=0.0125484704514077, P78=0.01358067158027406, P79=0.01452589815325243, P80=0.01718040641326345, P81=0.02093775338686881, P82=0.03416064138995377, P83=0.04185175408734609, P84=0.035933216337281235, P85=0.021229897491233017, P86=0.011779580391489106, P87=0.01244533970309512, P88=0.015013273551192889, P89=0.03277576094554188, P90=0.04180231477395486, P91=0.017093049108827696, P92=0.007664211622080238, P93=0.0015929316644677904, P94=0.0061973119511118755, P95=0.009272095818628793, P96=0.011772179531020747, P97=0.009938131067411286, P98=0.002021541230071461, P99=0.00033445562088842606, P100=0.00044276739342639227, P101=0.0004030092364104636, P102=0.0014315758850447283, P103=0.0016898390505839063, P104=0.0008147770691344817, P105=0.0004246414284663622, P106=0.0005726254465122245, P107=0.0005650288206584835, P108=0.0002931457611544126, P109=0.0005524637382498017, P110=0.004225822944036727, P111=0.005031922352593808, P112=0.0008036130762306707, P113=0.0006180418107529207, P114=0.0006559225787259228, P115=0.0015205505197856382, P116=0.0014073329786726272, P117=0.00045765424783254365, P118=0.0003466655873683827, P119=0.0005374183566108055, P120=0.0004095530719020576, P121=0.0004308343413794363, P122=0.0004008751107230333, P123=0.0002008124381005509, P124=9.256036197428562e-05, P125=0.0001306857493677955, P126=0.0004611298804168236, P127=0.00020689070720649526, P128=7.083174951718611e-05, P129=0.00010275404122741985, P130=6.434436646741018e-05, P131=0.00011321813967269837, P132=0.0001514929419296319, P133=0.00013803442334551122, P134=0.00027087885074669293, P135=0.0004926345061797847, P136=0.000178287426782765, P137=0.00014367597487093865, P138=8.810666257202368e-05, P139=3.89981057133505e-05, P140=4.7682897356228334e-05, P141=5.6309135356114886e-05, P142=9.712233007023033e-05, P143=0.00013313687626863532, P144=0.00016547720475482733, P145=0.00020400513242038086, P146=0.000223658917873706, P147=8.900426406274077e-05, P148=3.0769397974236034e-05, P149=2.5622295624584687e-05, P150=3.302465106972199e-05, P151=2.5648639243013587e-05, P152=3.1442354482677276e-05, P153=3.24096857560102e-05, P154=4.740620729679111e-05, P155=4.191683486220352e-05, P156=3.4570466149818855e-05, P157=2.7895861553577144e-05, P158=4.3286956541517684e-05, P159=9.527247518156405e-05, P160=8.570856330689316e-05, P161=6.76548470498698e-05, P162=3.853334180538127e-05, P163=2.3813471408456904e-05, P164=1.7933348579774104e-05, P165=1.5129312118597571e-05, P166=1.868407475663315e-05, P167=1.8757997948856086e-05, P168=1.4079450294545302e-05, P169=8.175602672690504e-06, P170=5.065089955658848e-06, P171=6.183610324202358e-06, P172=6.513264159874108e-06, P173=4.77520308350778e-06, P174=2.7023884887517616e-06, P175=3.879875918574659e-06, P176=5.260593870736237e-06, P177=6.8656657190881135e-06, P178=4.8043968944382354e-06, P179=4.91893762769107e-07),Row( P0=-719.2570298887571, P1=65.95794588158591, P2=-8.799352646034999, P3=5.438850287250822, P4=-9.40497564317709, P5=-8.87019198285138, P6=-7.370366295962456, P7=-1.9146878741459188, P8=-0.323735810202473, P9=-2.551802512065901, P10=-9.864208968536504, P11=-7.338149702966495, P12=-2.7600442465565633, P13=-6.238719578112443, P14=-4.206116281372609, P15=-6.438061371002477, P16=-1.625402700808808, P17=-7.178506590434467, P18=-5.266828570794507, P19=-2.587802765539263, P20=-9.113031877865932, P21=-6.587480925064262, P22=-3.6907460730640507, P23=-5.3807977416165595, P24=-7.356725238755314, P25=-2.90212453049343, P26=-6.299702992053323, P27=-5.693173012086668, P28=-4.455844459726922, P29=-4.213855908386634, P30=-2.913872709494843, P31=0.03737471791609361, P32=-2.8995349074219883, P33=3.367216383501255, P34=-0.11161329892053548, P35=3.5885783062357794, P36=0.5363620322970657, P37=2.594120703280682, P38=0.6730963188623804, P39=0.9937649518111106, P40=0.7183536648996884, P41=0.6938407840041211, P42=0.7264292390026579, P43=0.7283137715694588, P44=0.7452671740384558, P45=0.7218853378041694, P46=0.6940056617977473, P47=0.685888783307716, P48=0.7128147107060743, P49=0.7318967869357339, P50=0.7388314034671225, P51=0.7429848148042966, P52=1.1793514882849448e-07, P53=5.827572592862298e-07, P54=8.523342312686871e-06, P55=3.6849584405486744e-05, P56=0.0006610664721292926, P57=0.008655484552405827, P58=0.024327851817439405, P59=0.03071087863223295, P60=0.010749251523655592, P61=0.0015455036946648226, P62=0.0004343817061888716, P63=0.0005508352851378978, P64=0.0023395547797960266, P65=0.01253395852857563, P66=0.02241544279443205, P67=0.02252884076681604, P68=0.009809990999959578, P69=0.0033029620014356394, P70=0.0018089744581372034, P71=0.0019056611023091023, P72=0.002888833468985319, P73=0.004396468042332105, P74=0.0068105871433587935, P75=0.010120079305915105, P76=0.005165692536769039, P77=0.004026730194757164, P78=0.0055023974863508095, P79=0.005395454226090675, P80=0.005764800616974667, P81=0.005062304799012907, P82=0.0034804311928411983, P83=0.0045991692028887945, P84=0.008123113478022883, P85=0.005192459316605778, P86=0.005343787211090162, P87=0.008565216257918208, P88=0.012260991203900473, P89=0.0059878735932042595, P90=0.0038741655567182086, P91=0.007475395274497957, P92=0.0066304968853146025, P93=0.009378992959532543, P94=0.012925006333770124, P95=0.005599224324114822, P96=0.0024569403415482345, P97=0.003397970028054453, P98=0.0023839066245359276, P99=0.0003149249519317619, P100=0.000234559901201224, P101=0.0004273218614732305, P102=0.0006997809192143059, P103=0.0005101228519262492, P104=0.00023960954131952938, P105=0.0004948267613024395, P106=0.0004130521765332348, P107=0.00018446461395641057, P108=0.0011436444645056881, P109=0.002444204468515193, P110=0.003177707249416766, P111=0.004627455324840317, P112=0.0011826431997169397, P113=0.0009101042666903805, P114=0.0011250169296451796, P115=0.0005363193958288283, P116=0.0005027976049886901, P117=0.00046133630094182264, P118=0.000476881925631642, P119=0.00027550587526371356, P120=0.0003113638309166377, P121=0.00028894376987799117, P122=0.0004045985096496558, P123=0.0001866113938692956, P124=8.773962593269686e-05, P125=3.141535340886406e-05, P126=3.204981335134031e-05, P127=1.946210601620209e-05, P128=3.354826197275304e-05, P129=2.8916496550124236e-05, P130=2.876631507068919e-05, P131=6.129816393485962e-05, P132=0.00011916746807934679, P133=0.00013284722341700954, P134=0.00018289415969843502, P135=0.00013863838426744445, P136=0.00018526375620603965, P137=8.560132228842423e-05, P138=3.377577768024354e-05, P139=1.6602217289142247e-05, P140=1.749131931754951e-05, P141=1.7104732064552584e-05, P142=2.686150531928438e-05, P143=3.2458196215055685e-05, P144=5.953357775521218e-05, P145=8.30067299293313e-05, P146=6.928296197809535e-05, P147=5.0999402686708215e-05, P148=1.8030686264531586e-05, P149=1.7748236173861118e-05, P150=3.3078955696180564e-05, P151=6.41864042591205e-05, P152=0.0002835564695243797, P153=0.0009034960516865841, P154=0.0009089433644805795, P155=0.00036796273293355125, P156=0.00020940038366465023, P157=0.00010709889224373728, P158=6.71922797148377e-05, P159=6.856706655278732e-05, P160=9.370624917832306e-05, P161=7.274556596165647e-05, P162=5.7567953533226966e-05, P163=2.6722800775816678e-05, P164=1.6330993875910114e-05, P165=2.092991230533369e-05, P166=1.0376138394507263e-05, P167=1.0926859045417203e-05, P168=7.283779615134277e-06, P169=4.165283659066619e-06, P170=2.189670419823082e-06, P171=2.852255263196584e-06, P172=3.878734812839735e-06, P173=4.378968323735383e-06, P174=5.186333587064564e-06, P175=1.4729171752603668e-05, P176=2.8341348327925523e-05, P177=4.634141333061095e-05, P178=2.5589197179047577e-05, P179=1.3080197646505676e-06),Row( P0=-681.8845165812528, P1=70.76526726660812, P2=-11.203547987750873, P3=-0.24646714082519414, P4=-15.033158053602682, P5=-11.244490663978464, P6=-12.947773150083766, P7=-2.933870024059827, P8=-0.9816055635905101, P9=-9.044980079612444, P10=-9.969607165558433, P11=-4.590236045921236, P12=-6.242893345958054, P13=-6.677691904876594, P14=-6.015783813073877, P15=-7.126743418544233, P16=-2.0632026372035708, P17=-5.490695927688086, P18=-7.16201409607973, P19=-3.247098894080467, P20=-7.923103992447688, P21=-5.377794396680637, P22=-4.143835006175219, P23=-6.320037119909697, P24=-7.123460760715415, P25=1.2328313205982304, P26=-3.8202700775069403, P27=0.756571049405988, P28=2.591958298788318, P29=2.8749791663189797, P30=0.5247825856674182, P31=1.2225679976691552, P32=-1.1963571579833794, P33=0.679161852479235, P34=-0.6964832398051023, P35=-0.7993041391413249, P36=-2.4500123828034717, P37=0.1914736068722239, P38=-0.8188634404075845, P39=1.7133071766554158, P40=0.6505178112963407, P41=0.6571548775588782, P42=0.6787507279574353, P43=0.7252661583701355, P44=0.7533642577163572, P45=0.7325458743325762, P46=0.6720005021700378, P47=0.6339475267542684, P48=0.6597915679413867, P49=0.7078222389380838, P50=0.6995477668076703, P51=0.6529651110660384, P52=1.9813704955575008e-07, P53=9.490019962926475e-07, P54=3.7244387843190214e-06, P55=2.2878381005275158e-05, P56=0.0008290498202732861, P57=0.006357732756585012, P58=0.008510426002334485, P59=0.014094998294951986, P60=0.028078919082164068, P61=0.02901004685699806, P62=0.009003842578346986, P63=0.0011033866824483938, P64=0.0018405653137309724, P65=0.0040732653731210666, P66=0.004650813450782461, P67=0.007670300922394749, P68=0.009955785171783152, P69=0.01663875538907724, P70=0.025727135118567026, P71=0.015319609071705315, P72=0.004877887999252888, P73=0.0019402145771457858, P74=0.0012081269330981375, P75=0.0014214262246433724, P76=0.0017497764210252387, P77=0.0043608787630045635, P78=0.009424724976479549, P79=0.01766479831296178, P80=0.022871701087537014, P81=0.015490410040643136, P82=0.008589942305276927, P83=0.00659294967475422, P84=0.013281703202769661, P85=0.010374003900513745, P86=0.008725141274691896, P87=0.008626263607004402, P88=0.010885958826188168, P89=0.010131423400730385, P90=0.009674426993990872, P91=0.010626292031207763, P92=0.0050108958440549835, P93=0.0023685600189753634, P94=0.003491896237475151, P95=0.0035095375424916597, P96=0.002114196250046927, P97=0.0016416573050590361, P98=0.0011954430742770069, P99=0.0005006322764435624, P100=0.0005732843002629537, P101=0.0005742417212814344, P102=0.000462990617574087, P103=0.0002891902001653155, P104=0.0003354454378674326, P105=0.0007527292044255773, P106=0.0009787378506447412, P107=0.00035120354391354584, P108=0.00033788710350993477, P109=0.0004981008108284475, P110=0.0007667525292054973, P111=0.000830399528441518, P112=0.00044879545759039433, P113=0.0003019986248367709, P114=0.0002880301080379638, P115=0.0005225731068787036, P116=0.0005735237648760518, P117=0.0002688451232447277, P118=0.00016353252519642834, P119=0.0001272730523900582, P120=0.00017581205311826767, P121=0.00016472136551543806, P122=0.00027055966863409907, P123=0.00010639086269337868, P124=4.5246130277654604e-05, P125=0.00010571703402815016, P126=4.730980186908704e-05, P127=3.1216625743660264e-05, P128=4.273825807298578e-05, P129=6.394564454413409e-05, P130=2.651277368316917e-05, P131=4.383141120931987e-05, P132=6.006618269380244e-05, P133=8.49094960286849e-05, P134=7.50232993801476e-05, P135=4.517917324537859e-05, P136=2.6944239408149638e-05, P137=1.1247197099342768e-05, P138=6.37051323102583e-06, P139=5.912356311346987e-06, P140=8.139796360389802e-06, P141=1.682440817513004e-05, P142=2.5846762033302737e-05, P143=3.879736225557657e-05, P144=8.039722079222476e-05, P145=6.77089157140018e-05, P146=9.492867694234129e-05, P147=5.5176320474574275e-05, P148=2.8322325468790334e-05, P149=2.7188620091449418e-05, P150=3.173155151986108e-05, P151=3.113676088989211e-05, P152=4.243678485411464e-05, P153=7.28037237508538e-05, P154=0.00016096914222830805, P155=0.000382025480201618, P156=0.0011898695129904505, P157=0.0008304629321788694, P158=0.00020263979861237155, P159=0.00010962791467055956, P160=7.408059903324638e-05, P161=9.150732530815665e-05, P162=7.511199109669747e-05, P163=8.145986083609118e-05, P164=5.741422183494118e-05, P165=4.7941518187111595e-05, P166=3.4340581303256134e-05, P167=1.9483729914086736e-05, P168=1.0194387261206812e-05, P169=9.979547933434743e-06, P170=1.6298748814841607e-05, P171=1.9563469403555815e-05, P172=2.0275339623963807e-05, P173=1.745160145174035e-05, P174=1.3830119172640738e-05, P175=1.0195415650914545e-05, P176=8.170430212207541e-06, P177=7.810281737323351e-06, P178=5.8539127783700835e-06, P179=9.793469615642058e-07),Row( P0=-626.7227775289299, P1=52.357260150360894, P2=-17.478268743120335, P3=-1.9821949931705318, P4=-18.38292999513039, P5=-20.33940112029418, P6=-11.444518686835295, P7=-4.21280214095891, P8=-0.841660758694264, P9=-4.226545468024056, P10=-13.111817832248278, P11=-4.430631098025693, P12=-6.41692999637195, P13=-8.31743614957329, P14=-4.23924240852129, P15=-7.719821041459757, P16=-1.612845117423086, P17=-4.899093749869816, P18=-5.200179671194207, P19=4.6404171916034755, P20=-1.0279630163318347, P21=1.939071811490002, P22=4.489468486493712, P23=3.8148349477407932, P24=2.6211387824427765, P25=6.255521035783561, P26=2.6995275445415645, P27=4.941053973535192, P28=3.4788748363209985, P29=0.1661959190553377, P30=-1.148361183324859, P31=0.5341920808649102, P32=-2.395801128823157, P33=1.5856534457849936, P34=-2.4334527721394554, P35=-2.4071967959760987, P36=-3.1952660813122464, P37=-0.09643383406503388, P38=-0.2683413810131832, P39=1.2914639079087302, P40=0.6494472640296991, P41=0.6246316239499544, P42=0.6473056702245616, P43=0.6689253086853648, P44=0.7030925658018422, P45=0.6695311124154047, P46=0.6722683899000023, P47=0.6820158785731862, P48=0.7406151287864008, P49=0.7220984155135319, P50=0.6539494744808133, P51=0.6184335160207898, P52=5.507433907203936e-06, P53=4.217524718441509e-06, P54=7.611706178033209e-06, P55=1.5039347351727174e-05, P56=2.5136963354116976e-05, P57=9.944558420036442e-05, P58=0.0005702333225087356, P59=0.0020564602973116536, P60=0.01701331466322856, P61=0.07867039369090441, P62=0.11368400649301913, P63=0.07183736415963055, P64=0.043832959592963516, P65=0.033820808927611024, P66=0.01085259829040243, P67=0.0018649144787745704, P68=0.0018274519290923036, P69=0.0064291263695597884, P70=0.01472585827151432, P71=0.04098042164565521, P72=0.0628340748064185, P73=0.06256680045295583, P74=0.039817848362776985, P75=0.028717334038626974, P76=0.044814017677654305, P77=0.11884525783559402, P78=0.1425508880754855, P79=0.08877704354348397, P80=0.053984813024780656, P81=0.07142730745764773, P82=0.061014082387391415, P83=0.05882433793244468, P84=0.06913640593107015, P85=0.03888884641009381, P86=0.02282997579918701, P87=0.026702516857275545, P88=0.030481166560527333, P89=0.0319935758891086, P90=0.06820068287656118, P91=0.08002066516089566, P92=0.09726440640804813, P93=0.031097560650792604, P94=0.002246422397992507, P95=0.001113629342503841, P96=0.0009420681329058195, P97=0.0019767037237767094, P98=0.002005837680267943, P99=0.0014640767019809932, P100=0.0008267431596762393, P101=0.0005592110189321043, P102=0.000358196382188699, P103=0.0009559926916502218, P104=0.0007107831768280432, P105=0.0013190920599359915, P106=0.0028315557163850736, P107=0.0016315850073984493, P108=0.0021315349441280414, P109=0.0012524309800204993, P110=0.0017162758369901052, P111=0.0021231768748427777, P112=0.0016815184702981816, P113=0.0029816115621203003, P114=0.00484897868421269, P115=0.0026394409879299235, P116=0.005191154716286891, P117=0.0016569211540237738, P118=0.0024473802797157188, P119=0.0034575856688611674, P120=0.001887303795401618, P121=0.002848812703191377, P122=0.001336657351091042, P123=0.0006804876061003644, P124=0.0006395314724094137, P125=0.0003992548582118896, P126=0.0005821700982155447, P127=0.0003513689827200835, P128=0.00024703311412490236, P129=0.00035653028785231325, P130=0.00025649322312660297, P131=0.0006040046779274565, P132=0.0004506729053375914, P133=0.0008371678638668594, P134=0.0004911315712322128, P135=0.0006555988165728831, P136=0.0001900835125730455, P137=0.00012997965221574944, P138=0.00010348413323714407, P139=8.078403697465247e-05, P140=9.426997971725147e-05, P141=0.00010209513999651014, P142=0.0002557375059580798, P143=0.0005024044277460643, P144=0.0012209138564513154, P145=0.001225885850250049, P146=0.0004801986663100075, P147=7.672751599556529e-05, P148=3.1889681327565905e-05, P149=1.994675379718426e-05, P150=1.93072486040523e-05, P151=2.2426475309608612e-05, P152=1.992342075151563e-05, P153=2.5243165482998068e-05, P154=3.0684154967815516e-05, P155=3.384240701321616e-05, P156=3.856591081904536e-05, P157=3.1942049339063515e-05, P158=5.839359792602447e-05, P159=9.018989313257767e-05, P160=0.00011410000683098553, P161=0.00011647354436433514, P162=7.091475840421088e-05, P163=4.110736254410099e-05, P164=2.8755139256493677e-05, P165=1.5176099331665e-05, P166=1.1129619411205908e-05, P167=1.4451007399339157e-05, P168=2.060650070775022e-05, P169=1.4936735414735118e-05, P170=1.706768612039208e-05, P171=1.0936912587881972e-05, P172=1.1927741851083701e-05, P173=1.6481378434863135e-05, P174=1.2543366181698684e-05, P175=7.650249915386208e-06, P176=8.74628523854976e-06, P177=1.300331557813698e-05, P178=1.1084649892930715e-05, P179=1.4246773547177964e-06),Row( P0=-547.7961505980276, P1=41.35820434002668, P2=-9.187953554858705, P3=-18.533030480266795, P4=-18.742585309884863, P5=-14.134559231197684, P6=-20.455969490161753, P7=-7.569232395988357, P8=-10.351145456930787, P9=-18.270102541464983, P10=-10.508903190863178, P11=-11.49000690239154, P12=-9.366278270549444, P13=-2.942021731609611, P14=-5.998380554664192, P15=-0.5263992926623089, P16=4.724177668893698, P17=0.000803032975854399, P18=0.6947360914486042, P19=4.043744138559731, P20=-4.0212594041584335, P21=2.729370496788366, P22=3.981496000009737, P23=3.8058916860839735, P24=0.7964275761249331, P25=2.150628905591483, P26=-2.229973024222092, P27=1.3289940138503678, P28=0.5826200772756505, P29=2.5510522688890727, P30=0.2502316102945138, P31=3.2215053780654777, P32=-2.8959250504754097, P33=2.532011174637097, P34=-2.3399819680329186, P35=-0.6569461697143951, P36=-3.358840715855131, P37=-1.7216220453696813, P38=-1.9284379231397972, P39=0.7807692034142975, P40=0.6503574359538569, P41=0.6694585531035895, P42=0.6564306244594217, P43=0.6455326068363817, P44=0.65090796854706, P45=0.6596863526375276, P46=0.6960744205516217, P47=0.647819897755588, P48=0.641552574113195, P49=0.5931973623561136, P50=0.5405796629147659, P51=0.5786619408138868, P52=5.851221971301135e-07, P53=3.8812895649725025e-06, P54=1.483076320648631e-05, P55=8.478409462568247e-05, P56=0.000132187904981449, P57=4.886465521139623e-05, P58=0.00026195049161954, P59=0.002381669264313849, P60=0.010048471873841797, P61=0.026208117983171242, P62=0.07032133809072622, P63=0.09948696694064468, P64=0.08682408721316504, P65=0.16231044820349563, P66=0.23247353492318795, P67=0.2928489896357399, P68=0.5044400386562972, P69=0.5684928493672291, P70=0.2186400280525448, P71=0.13330466228857263, P72=0.0696983281463162, P73=0.017936161136754107, P74=0.02601189622531799, P75=0.030410918165858863, P76=0.018438855761913727, P77=0.042031808030296686, P78=0.08150982445070051, P79=0.09283738969809939, P80=0.0815124796412017, P81=0.09241438661059542, P82=0.15161924435363153, P83=0.19620569813208227, P84=0.31660503034584087, P85=0.381350634213389, P86=0.22649504321895972, P87=0.19179378986955492, P88=0.1235678651801335, P89=0.04896509873227052, P90=0.06975712294895169, P91=0.09645401773493707, P92=0.10264363470381321, P93=0.06949970077332854, P94=0.15609400092820583, P95=0.1896765027700385, P96=0.1410758621481948, P97=0.274071739787893, P98=0.48649642579855196, P99=0.1642702913201158, P100=0.07033847095624672, P101=0.008802001548996592, P102=0.002316777732951715, P103=0.003542020089814433, P104=0.005999793464825094, P105=0.009554729082766535, P106=0.030646148926756652, P107=0.02965536338907195, P108=0.007811381701353166, P109=0.01383904329419578, P110=0.05651620866096655, P111=0.062122946517358, P112=0.010004635991763221, P113=0.00967415079253499, P114=0.008158928968934238, P115=0.005399786377943055, P116=0.0037492800736889367, P117=0.0023625994411801943, P118=0.004986674409595658, P119=0.00837969052223931, P120=0.004752154051181414, P121=0.0010833347745962962, P122=0.0013090161803796499, P123=0.0019288071608664253, P124=0.0064661933522922625, P125=0.006572334585416133, P126=0.0008649597510460409, P127=0.0018913007614346276, P128=0.0019282195440452998, P129=0.0015569904389911573, P130=0.000871947997478771, P131=0.0010597320338390924, P132=0.0012526223978767946, P133=0.0014921022516214341, P134=0.0014162486145340783, P135=0.0016229257240875134, P136=0.0005499891678674185, P137=0.0003999281062440066, P138=0.00012245179855329368, P139=3.5370909802661745e-05, P140=2.6697588855435474e-05, P141=3.637944710314133e-05, P142=6.135171415335469e-05, P143=0.00011897131676201798, P144=0.00022939631350966035, P145=0.0004335859710257741, P146=0.0004784415543110654, P147=0.0010751764080027635, P148=0.003181358305218366, P149=0.0029103120811535678, P150=0.0052390071659489746, P151=0.0020804443198757074, P152=0.001610094038116421, P153=0.0016881613895237367, P154=0.0009683184852570685, P155=0.0008367439072948496, P156=0.0007866191391172934, P157=0.0005672396717130838, P158=0.000581416273081645, P159=0.0007354489829756256, P160=0.000671665505748015, P161=0.0006196024475153534, P162=0.0006395688241605732, P163=0.0008038552344251535, P164=0.0006034896783255288, P165=0.0005307177886829528, P166=0.0004938611293843076, P167=0.0004811318294881975, P168=0.0004619700951584579, P169=0.00045960371452350823, P170=0.0005137646728331838, P171=0.00044656503443787416, P172=0.0004750886249073632, P173=0.000645491934905398, P174=0.0008573508533657676, P175=0.0011144494102968716, P176=0.001306110208558822, P177=0.0005874930344993088, P178=0.00015699043412418105, P179=1.1565349215830497e-05),Row( P0=-665.6577085502449, P1=56.65734567515667, P2=-8.283363954364745, P3=5.064178126194238, P4=-17.09384034680909, P5=-10.849900833208272, P6=-8.806252860130467, P7=-3.82738357913081, P8=-0.7656188472519881, P9=-2.0012673529355025, P10=-12.764555788056771, P11=-6.020567156881007, P12=-3.976340454412616, P13=-7.68262110241715, P14=-2.5043772331907603, P15=-7.827377424779726, P16=-2.5059745615725335, P17=-5.893370770153336, P18=-6.595045656189592, P19=-1.5111748261227795, P20=-9.156032767295326, P21=-5.2743092084387, P22=-3.9006583562186288, P23=-4.948160238878877, P24=-5.5303035307557735, P25=-0.984091580054028, P26=-2.899237413122899, P27=-0.6186015091202972, P28=1.8574766386806913, P29=0.9299457682629557, P30=0.6086038193144649, P31=3.9010501356405265, P32=-2.298235165921883, P33=2.8652375083959347, P34=-1.5919094347356846, P35=0.8358361378537571, P36=-2.6536783716653014, P37=-1.2081898474262083, P38=-2.369428416599579, P39=-1.3010939417640048, P40=0.7472364980576272, P41=0.7364586666149094, P42=0.7176189358530164, P43=0.7350596409943186, P44=0.7417709643442834, P45=0.6846051844730613, P46=0.6361586942810669, P47=0.6536100883888384, P48=0.6386797265796629, P49=0.6773536196177654, P50=0.6987893526803917, P51=0.690829675208327, P52=1.2009365403006986e-07, P53=1.0100543252995698e-06, P54=1.197713322288993e-05, P55=3.6455893022327726e-05, P56=0.00015307414356962798, P57=0.001238151753835368, P58=0.006198518113628783, P59=0.024226694795040124, P60=0.04917170819409426, P61=0.037224868710377036, P62=0.011966733917214572, P63=0.0022163421929049204, P64=0.0006832473419037381, P65=0.0023351086266128035, P66=0.007734695246434725, P67=0.01986529031753751, P68=0.02948140950381748, P69=0.04913208483380816, P70=0.06025237445518368, P71=0.02242639350065498, P72=0.013032958024299868, P73=0.011652993112261829, P74=0.005790348944280898, P75=0.006562729756384245, P76=0.00914102433954715, P77=0.010972140441372303, P78=0.020162147348284018, P79=0.04971710845252226, P80=0.029339673804410236, P81=0.009061642391763543, P82=0.006754697752469341, P83=0.016482778383297043, P84=0.02904168839326518, P85=0.01723411611440878, P86=0.005605556448088754, P87=0.013299080008835739, P88=0.030834067751947792, P89=0.01931101325097977, P90=0.01796338145500346, P91=0.014266187522493628, P92=0.010077492947706993, P93=0.0020251392782068833, P94=0.007536510355443661, P95=0.008955483161406362, P96=0.011726966217506636, P97=0.006956679199611836, P98=0.0005114775130311028, P99=0.00021822762925715575, P100=0.0007140063614959095, P101=0.0005601222383431774, P102=0.0009835721570683928, P103=0.00031030171538435865, P104=0.00038004326621469384, P105=0.0007810991372170284, P106=0.0004479356834321173, P107=0.0007629822334205863, P108=0.0005560011636921987, P109=0.0017077948442957552, P110=0.0014758725900750737, P111=0.00271436185572184, P112=0.0015192672552974019, P113=0.0006002294960197978, P114=0.0003736062998988669, P115=0.0008827191749846332, P116=0.0008332118806534917, P117=0.0004537523371790529, P118=0.0007798668560758753, P119=0.0003293523206420844, P120=0.0003926092435779789, P121=0.000568313947900051, P122=0.0002795192334427409, P123=0.00040231233604849734, P124=0.00019697467495827548, P125=9.621932751947258e-05, P126=7.88071174134508e-05, P127=0.00017413790889468514, P128=7.0880180790337e-05, P129=0.0002149023521462832, P130=7.751653968175022e-05, P131=0.00010913651440693745, P132=0.000259681230060406, P133=0.0003773168652404288, P134=0.0004267541017025013, P135=0.00032587715679498083, P136=0.00021026717967757722, P137=0.00014642200514542966, P138=9.43945076831295e-05, P139=5.402803655311795e-05, P140=6.682490133575973e-05, P141=0.00010389333455868805, P142=0.00020526066749405525, P143=0.00027482395016300627, P144=0.00025629242055125526, P145=0.0003448909088145327, P146=0.00033814523441044064, P147=0.00022105469807783717, P148=6.883843031549704e-05, P149=0.0001807432439372505, P150=0.000253538934962614, P151=0.0002177233182465369, P152=0.00026549602603755595, P153=0.00036958403862793197, P154=0.0003126339264429308, P155=0.00026024618826860676, P156=0.00027453588305034884, P157=0.0001742674750210374, P158=0.0001742883587421861, P159=0.0002736796329356542, P160=0.00028365238881877564, P161=0.00018973573347965425, P162=0.00010633056021807543, P163=5.01616834441772e-05, P164=2.1983632019416548e-05, P165=1.763203652080915e-05, P166=2.1050803083845257e-05, P167=4.2482712428542434e-05, P168=3.784361501414476e-05, P169=2.892615593022693e-05, P170=1.0886238599433477e-05, P171=1.2449689577637195e-05, P172=1.9015282624979545e-05, P173=1.8938048946335233e-05, P174=1.4447111829992695e-05, P175=1.8239267982393687e-05, P176=1.8037174919285894e-05, P177=1.6156049829244686e-05, P178=8.053664161017825e-06, P179=8.635001142275511e-07),Row( P0=-748.3474820536134, P1=72.7125800492921, P2=-6.337924735019951, P3=5.084748253249037, P4=-8.095854824132616, P5=-10.458179594187754, P6=-5.005633965918364, P7=0.04644022649008506, P8=-2.144281040721218, P9=-1.632041192294031, P10=-6.876671109151116, P11=-6.224185588905996, P12=-1.9720178563696522, P13=-5.391663191074558, P14=-5.671743662916773, P15=-4.997335291552669, P16=-1.4580974020909188, P17=-5.330695271093386, P18=-6.108442150361441, P19=-5.070707143905034, P20=-6.758558790096035, P21=-6.820402063735948, P22=-3.812899905308305, P23=-3.3963434965367356, P24=-9.2567111854106, P25=-2.646754824034905, P26=-8.028285911891064, P27=-5.39374704343407, P28=-3.723999412619901, P29=-4.867767549625234, P30=-4.034431967126056, P31=-2.0326926623687243, P32=-3.64018574597502, P33=0.5446973569243504, P34=-2.58195679961306, P35=0.694054883563755, P36=-1.9361446214197389, P37=1.7177610618759076, P38=2.5359067865255396, P39=3.9857386406005277, P40=0.736802011323064, P41=0.6799779849284386, P42=0.7098963621349452, P43=0.7512830641631218, P44=0.7440539255447897, P45=0.7297310156434238, P46=0.7410628984968911, P47=0.7408460905614864, P48=0.7990267914130876, P49=0.7629022936019282, P50=0.6978711814284531, P51=0.7060647424883733, P52=2.669851215447607e-07, P53=1.6096453054875409e-06, P54=9.43455683441771e-06, P55=3.311383493463643e-05, P56=0.001092799554223573, P57=0.01240105468668071, P58=0.015853493405473797, P59=0.012828309918432899, P60=0.0030489865962628026, P61=0.00034612642346193835, P62=0.00023818224591602962, P63=0.0006830781135436343, P64=0.00254197218751349, P65=0.004658925894213994, P66=0.006447292852476097, P67=0.009151792439427393, P68=0.0031455236182395332, P69=0.0012204913934391463, P70=0.0020213862350829427, P71=0.002224794124214503, P72=0.002050502262002772, P73=0.0018374550415492241, P74=0.0042116967507791064, P75=0.006841276769815175, P76=0.002313351847517106, P77=0.001297167615367578, P78=0.00193350753217617, P79=0.0022342460677774404, P80=0.0009005793510540929, P81=0.0005799223717288892, P82=0.0021631158722909024, P83=0.004810907437405987, P84=0.0045756033245827, P85=0.0027965806458799545, P86=0.0011370213205777128, P87=0.0007588941718281233, P88=0.0013813034973983967, P89=0.0014155493745431071, P90=0.0010710391576567292, P91=0.00034149839058842844, P92=0.00039916925168498996, P93=0.0007375766422867655, P94=0.0017870928054330867, P95=0.000726520395579862, P96=0.00022456526036885045, P97=0.0003458082626304448, P98=0.000626851098358649, P99=0.00027707548065454237, P100=9.94080081630331e-05, P101=4.618945488069229e-05, P102=6.789767221777904e-05, P103=7.442590571035758e-05, P104=7.776301697258465e-05, P105=0.00010990497506340835, P106=0.0002220380019793679, P107=0.00010785308384346909, P108=7.313412055382316e-05, P109=0.00021272123819117148, P110=0.0003410593704410154, P111=0.00022712650352707994, P112=7.943737098913258e-05, P113=8.124138694258444e-05, P114=4.632823406326141e-05, P115=0.000117221136626874, P116=6.554856333641081e-05, P117=5.926188231223764e-05, P118=9.109214025513708e-05, P119=3.4312619178951735e-05, P120=7.252970463581444e-05, P121=5.754842746287437e-05, P122=7.705221645805746e-05, P123=9.885256965364154e-05, P124=4.452276804586107e-05, P125=8.928154109465194e-05, P126=4.616849872068699e-05, P127=1.7199149174276694e-05, P128=1.207586003425915e-05, P129=2.7154183472294206e-05, P130=1.9690948422701373e-05, P131=2.5701273450984333e-05, P132=5.1755173637158656e-05, P133=6.20642273712257e-05, P134=7.116236355846492e-05, P135=2.3857653653711214e-05, P136=1.2857585372491987e-05, P137=1.1713543032830594e-05, P138=1.1750602119662558e-05, P139=4.544359347679995e-06, P140=5.687776565206444e-06, P141=6.619163846058374e-06, P142=7.6288707417152325e-06, P143=1.206694637470611e-05, P144=2.0892453041794755e-05, P145=2.019671448587191e-05, P146=8.396708556861464e-06, P147=2.637545807555104e-06, P148=1.2453386852253737e-06, P149=9.807567777761103e-07, P150=1.989230891196726e-06, P151=3.956276775084291e-06, P152=8.763196783725524e-06, P153=1.3356393950751026e-05, P154=1.5738697888205448e-05, P155=1.1306219337937397e-05, P156=5.810509460433101e-06, P157=7.65949579054497e-06, P158=5.4189953275189955e-06, P159=7.998876113964307e-06, P160=6.281196465060581e-06, P161=3.677056075238703e-06, P162=4.244203003594683e-06, P163=2.0346761979864746e-06, P164=1.1356575102980436e-06, P165=1.2602954186699064e-06, P166=1.380252171248186e-06, P167=1.7070393133136499e-06, P168=1.5474540487194844e-06, P169=1.393381943242779e-06, P170=9.410431649336896e-07, P171=5.164752409145285e-07, P172=5.041280037776663e-07, P173=5.440889361324292e-07, P174=5.806102872559407e-07, P175=9.4371341205081e-07, P176=1.8467673437435199e-06, P177=1.9062013606589829e-06, P178=1.260106024379968e-06, P179=1.136597219591416e-07),Row( P0=-667.4990156021314, P1=62.39645489729108, P2=-14.613415336323769, P3=2.795401150046719, P4=-15.029700119398646, P5=-13.87498582175908, P6=-7.4922479194396345, P7=-3.779025371348517, P8=-3.3333537442503136, P9=-3.785284741805876, P10=-11.331450691481207, P11=-4.266582455810501, P12=-4.9486109085862955, P13=-8.039644190034487, P14=-1.6578716700349831, P15=-7.332828534481365, P16=-2.9913513978355537, P17=-5.625112794219168, P18=-8.014215151436684, P19=-0.09638762079733226, P20=-7.629171796772704, P21=-2.8335051804558007, P22=-0.4021800150042695, P23=1.0053792477165178, P24=0.5792016515942305, P25=4.939442421200185, P26=-0.27143688393026866, P27=1.9793763924072816, P28=3.265545066725861, P29=1.4305869521036663, P30=1.2133124194247045, P31=0.850055310360513, P32=-4.059143302706567, P33=-0.7853791245291915, P34=-4.570951336742024, P35=-0.8856648328518357, P36=-2.4366470612174256, P37=-1.2492437621035641, P38=-2.560758830583419, P39=-3.165050766413981, P40=0.7123963034689764, P41=0.7368183741472772, P42=0.7738981212637, P43=0.768121428463304, P44=0.7372154746329964, P45=0.7517195849749411, P46=0.7572149886921803, P47=0.7071818081402119, P48=0.6689451687202612, P49=0.6975193662187706, P50=0.698938054654209, P51=0.6884678680804333, P52=3.3213893421707463e-06, P53=3.687933376843952e-06, P54=1.3812944821998853e-05, P55=3.2629464546855595e-05, P56=9.656295368444222e-05, P57=0.0006498361365503326, P58=0.0009956278834343567, P59=0.00948508702207366, P60=0.046578946062500355, P61=0.06243108258290676, P62=0.06848332663729714, P63=0.04270217520360813, P64=0.007332976495601654, P65=0.0017215281084552547, P66=0.0017947772208772794, P67=0.003288270246612818, P68=0.01763114345443418, P69=0.04404602998152918, P70=0.05934789161211231, P71=0.04007090639323716, P72=0.035583436231003664, P73=0.03979250643638037, P74=0.03665091513499728, P75=0.02181233506696375, P76=0.007195492278498753, P77=0.008469427706136432, P78=0.01861669844135851, P79=0.03451191947945036, P80=0.03332248378755776, P81=0.02494691977778805, P82=0.02430140320490993, P83=0.028809056816886967, P84=0.04180734853169536, P85=0.03910623147595118, P86=0.04892248944452552, P87=0.07322057538326053, P88=0.05035183351038023, P89=0.00724768743346137, P90=0.0022187739602181396, P91=0.0020933758765939335, P92=0.002869119607478685, P93=0.008860511350657923, P94=0.032513965882203316, P95=0.015777436361274367, P96=0.00733601472529887, P97=0.004507067125407718, P98=0.0006798327622295115, P99=0.001263320265211523, P100=0.001741518182990676, P101=0.0011796513623235434, P102=0.002635478798045915, P103=0.00177972360278486, P104=0.0011474122466048247, P105=0.0008861511695205606, P106=0.001612342372578397, P107=0.0011053835887383605, P108=0.0013014527954329363, P109=0.0017582999342024772, P110=0.0017216512503047164, P111=0.001735909789350614, P112=0.0007259083718272317, P113=0.0003860810108789271, P114=0.0007851626822481205, P115=0.0026488027579105403, P116=0.001207105011362328, P117=0.0003866191144964978, P118=0.0013100659202970163, P119=0.002527149926265256, P120=0.0007951272478048908, P121=0.0007233700075106823, P122=0.0013845838683438254, P123=0.0002642950963187198, P124=0.00028201948039216534, P125=0.00038752289303403464, P126=0.0002554690112145982, P127=0.00014192792273955173, P128=0.000173307219118324, P129=0.00025253971027204043, P130=0.00029854661499397123, P131=0.00031386747933155867, P132=0.0004951593464458313, P133=0.00034607113204501976, P134=0.0003135324491202681, P135=0.00024817802525942356, P136=0.00015198345061531856, P137=0.00010911788923580234, P138=7.074260683315884e-05, P139=5.6234455389044444e-05, P140=8.577103088511848e-05, P141=0.00012865406009617312, P142=0.0002990232411413195, P143=0.0005054601909682559, P144=0.0006135271687930057, P145=0.0004312738150152056, P146=0.00033537566011664766, P147=0.00013575325183678855, P148=3.71411444478837e-05, P149=3.818475929515998e-05, P150=5.7575767432011714e-05, P151=4.051101797065236e-05, P152=4.9276235853683325e-05, P153=4.9558318302017396e-05, P154=6.228173296917221e-05, P155=4.893625017280002e-05, P156=4.686401003509521e-05, P157=4.682041485366296e-05, P158=3.459837402415891e-05, P159=1.820175672754416e-05, P160=2.56243237563202e-05, P161=4.735868183476033e-05, P162=2.7642034817719902e-05, P163=1.8462182877673366e-05, P164=8.508701009368455e-06, P165=6.28977400525816e-06, P166=7.819784294861261e-06, P167=9.965911708233118e-06, P168=5.976907942180448e-06, P169=4.587156155874383e-06, P170=7.792690770446514e-06, P171=3.460711724013746e-06, P172=3.5277413094592815e-06, P173=3.684668170627481e-06, P174=4.184874097982749e-06, P175=3.560153757511366e-06, P176=4.615985759774072e-06, P177=4.3327257992328e-06, P178=2.1980347183664196e-06, P179=4.3987572076231237e-07),Row( P0=-517.5437826045071, P1=35.2792522586166, P2=-25.515586459155152, P3=-17.554815831605737, P4=-18.890207739378454, P5=-14.763038733005365, P6=-18.86857836875523, P7=-5.455760757416312, P8=-3.9789984303709485, P9=-11.070131735435709, P10=-12.568805006774026, P11=-5.36573658853859, P12=-1.6373657047620098, P13=-1.0449866004224764, P14=0.046861596914563396, P15=5.16963267472315, P16=5.5081913730557845, P17=4.897815465805629, P18=2.6862384339800296, P19=1.6709874755056073, P20=-0.4525483708300466, P21=2.6612156687621082, P22=-1.4531811654119016, P23=-2.304475465200242, P24=-4.476382708152168, P25=1.4158117346304346, P26=-1.283320768992498, P27=0.3585794253404018, P28=-2.200812958011404, P29=-0.2471720576819428, P30=-0.07638830245220549, P31=-0.8498290301835193, P32=-2.3525397032271176, P33=0.5093759257466706, P34=-2.4254649399184554, P35=-1.2766694354855401, P36=-1.6746350465086288, P37=0.47928361705754247, P38=-0.5577639116752297, P39=1.8117930749045978, P40=0.5889015607013888, P41=0.5330292137751904, P42=0.5789258279111031, P43=0.6064652651029938, P44=0.6742527660667114, P45=0.6711348441107828, P46=0.6230692651452175, P47=0.6234592519917226, P48=0.6839265527717855, P49=0.6905739230241389, P50=0.6583981359575616, P51=0.6341479423344106, P52=2.425570623823923e-05, P53=2.941135193323736e-05, P54=1.4219472663807783e-05, P55=2.6648320520205032e-05, P56=4.963855736515873e-05, P57=9.89488797848246e-05, P58=0.00014004597876134204, P59=0.00019920973941803897, P60=0.0004275177551113818, P61=0.0025001676797991815, P62=0.012764665737079364, P63=0.028416791832094616, P64=0.0393943532155661, P65=0.09568406559363869, P66=0.18243496020790934, P67=0.2383120677065249, P68=0.2485394720617747, P69=0.26862933720715465, P70=0.303795457858974, P71=0.38085368875924647, P72=0.4856846058865337, P73=0.21711768911109652, P74=0.014371251111651396, P75=0.003555802797422877, P76=0.0037512045797614833, P77=0.00992263613947631, P78=0.020164131691124944, P79=0.02785737471744744, P80=0.03891117164559285, P81=0.05093128760106061, P82=0.07424790877770537, P83=0.12110659168777238, P84=0.19240913006093438, P85=0.17992763465723463, P86=0.34092169892349933, P87=0.7639302330380106, P88=1.1036009556626876, P89=0.7037849792131248, P90=0.6250971768143772, P91=0.1865627639329854, P92=0.15150727454435375, P93=0.06852682361104959, P94=0.09802357083873353, P95=0.09023370347332908, P96=0.06761933462189014, P97=0.08847295801604019, P98=0.1264779876013299, P99=0.08722219055292417, P100=0.16705804720003367, P101=0.07408414704716654, P102=0.013706745310435734, P103=0.0049983665396251, P104=0.006133310287460973, P105=0.00482541864070586, P106=0.005624415039140777, P107=0.014962862614085746, P108=0.020256647898584126, P109=0.010416588021848779, P110=0.014280685336532014, P111=0.009576390727911381, P112=0.006358425425282171, P113=0.008748022796176569, P114=0.009968133162395056, P115=0.006124821587404607, P116=0.002175745433149926, P117=0.00207153240119838, P118=0.005225450846588587, P119=0.007277327239819906, P120=0.004971195132627857, P121=0.0029331264545578014, P122=0.0031731581687725607, P123=0.0018100738868695233, P124=0.002156440105711682, P125=0.0028021793912207035, P126=0.001233985213686547, P127=0.0005658314516680613, P128=0.002194091242641546, P129=0.00912942110870043, P130=0.009886339446145148, P131=0.0038236937726284855, P132=0.005755828732467278, P133=0.0029024543221776947, P134=0.001381922486805832, P135=0.000883688697247734, P136=0.0027653669737691695, P137=0.005489782883071937, P138=0.00737162458131109, P139=0.0013031457065183513, P140=0.0011334376173183832, P141=0.0006079194714784992, P142=0.0006352751748765734, P143=0.000619711441162936, P144=0.00040375424934344626, P145=0.0004594578868407172, P146=0.0004882297605742694, P147=0.0008395567598662664, P148=0.0007171724090607928, P149=0.000982805460724945, P150=0.0008306750575314753, P151=0.0007341757813937544, P152=0.0006741375456110057, P153=0.0005698600042360771, P154=0.0005166879407060872, P155=0.00033542763896124867, P156=0.0003971107048745805, P157=0.0005401536342649811, P158=0.0003633899876853396, P159=0.0004514547705276996, P160=0.0007788601255603818, P161=0.0009501961197873305, P162=0.001132426652882439, P163=0.001076486196382684, P164=0.0007381698893779391, P165=0.0005532776977899828, P166=0.0008115795630799635, P167=0.0012426556041547918, P168=0.0012266064807101016, P169=0.0008280058402238393, P170=0.000764283368671384, P171=0.0006279100229517109, P172=0.0004073624043770907, P173=0.0002630624114097591, P174=0.0002485216893413672, P175=0.00025137827099963204, P176=0.00023614579594447816, P177=0.0001660775156960431, P178=7.379248362934252e-05, P179=9.360857616502774e-06),Row( P0=-464.95914333514133, P1=45.839434150251236, P2=-21.20990674529913, P3=2.385227937782218, P4=-9.574380506075459, P5=-10.830197817631765, P6=-12.332404115504241, P7=3.88031165243661, P8=7.286783541476615, P9=0.9531392768423195, P10=-8.946358046778947, P11=0.28439673290939504, P12=1.397772967646491, P13=-1.6889105181836432, P14=5.462643282619943, P15=8.261821169052514, P16=10.10744834488996, P17=5.54550660235966, P18=-1.7647910288629531, P19=8.23303047546393, P20=3.0835357966240453, P21=1.0121527571492173, P22=3.4845573489590684, P23=2.447495655228574, P24=0.9438036031368461, P25=4.076930011821482, P26=-1.162989989721689, P27=1.5790577248189674, P28=0.3327490357893891, P29=1.7343787600765919, P30=0.19425919278351758, P31=2.3988030051491744, P32=-3.312882264100469, P33=-1.4220437963677084, P34=-4.019633751191801, P35=-2.4078028924191264, P36=0.6834154571368934, P37=-1.7483573032092186, P38=-1.609230379977983, P39=0.18828567669457927, P40=0.6213626444135316, P41=0.5996731367780442, P42=0.6385490551075921, P43=0.629425155366994, P44=0.6481445924176105, P45=0.7234692708246014, P46=0.731105040999074, P47=0.631811194041492, P48=0.6628737401037843, P49=0.6894141133057614, P50=0.6563971770635142, P51=0.63837863970556, P52=0.008386984608747296, P53=0.009356950035674187, P54=0.005860596099975734, P55=0.0021116250315150686, P56=0.0008892244148215757, P57=0.0003650677130142745, P58=0.0002570087207841483, P59=0.0008874413173723143, P60=0.01589210940371283, P61=0.08109279473924992, P62=0.19629536920623722, P63=0.23285080623627988, P64=0.2038999808443285, P65=0.43525452420125166, P66=0.6719615204350091, P67=0.5736529536832707, P68=0.8683724566986949, P69=1.2941637030328055, P70=1.2725987441140827, P71=0.7124125792880883, P72=0.16083621817810834, P73=0.053749174031165775, P74=0.026742713671334798, P75=0.016883158344909607, P76=0.02451805283933605, P77=0.09867496030923639, P78=0.24830682200855378, P79=0.39144241975172744, P80=0.6570145956004951, P81=0.7235187018108732, P82=0.47175037755591126, P83=0.31892467614343195, P84=0.5296356846227721, P85=0.49262157708584564, P86=0.7334419711508711, P87=0.9828784734396657, P88=0.7609224511160484, P89=0.7988816007413755, P90=0.8466819543702095, P91=0.2875306880515712, P92=0.12444101630216325, P93=0.22227377908203239, P94=0.3920636058449954, P95=0.09197334534595454, P96=0.020164758288488443, P97=0.03365854251243052, P98=0.03871538924050519, P99=0.027127858941105285, P100=0.08197831798830865, P101=0.19432786891017517, P102=0.25006118358051027, P103=0.20888240978315983, P104=0.07363178886080786, P105=0.17104685328537475, P106=0.11026295633852229, P107=0.05034674594457442, P108=0.10845108081920324, P109=0.12162552184755714, P110=0.24339093533710454, P111=0.04174063237210401, P112=0.02687248664343396, P113=0.017982300710776455, P114=0.027501157973705174, P115=0.054145126083751445, P116=0.034273309502358955, P117=0.029551812233344504, P118=0.033665364192766445, P119=0.04736492094546686, P120=0.057077017181806286, P121=0.0054748289804709485, P122=0.012966897280880598, P123=0.010218062098644756, P124=0.010079714974083885, P125=0.007302612375237698, P126=0.00630764402149638, P127=0.007291738416075249, P128=0.004066324552379835, P129=0.0030877507126488395, P130=0.006029854359938876, P131=0.008662996953960169, P132=0.009578942091878517, P133=0.01626925841923819, P134=0.010254602987474524, P135=0.0036469496518210113, P136=0.003288827749148476, P137=0.0017472716060847645, P138=0.0023554629657713167, P139=0.002179982974534319, P140=0.002521593761858925, P141=0.005735348365184961, P142=0.00913767012525253, P143=0.01500916007991153, P144=0.016722706103880082, P145=0.0190185288319096, P146=0.005863825239835013, P147=0.0034020851401904933, P148=0.0023073875726293343, P149=0.0008841226036899716, P150=0.0014177191585230148, P151=0.0013498011145102514, P152=0.0023204591271375357, P153=0.001133899583835401, P154=0.0014912931336291237, P155=0.0014633175648254642, P156=0.0016361021322419842, P157=0.0011782933758671353, P158=0.0013983509832283083, P159=0.001426600596888975, P160=0.0009192755690653586, P161=0.0005574496396436804, P162=0.0007140669176383583, P163=0.0008058533592099739, P164=0.0007252399052499554, P165=0.000546708682109351, P166=0.0004546723452872404, P167=0.00029097791595245726, P168=0.0003398633335109725, P169=0.00028381009615560166, P170=0.00026508653187167977, P171=0.0002660261288545896, P172=0.0004144788647856269, P173=0.0004111803358168152, P174=0.00023531477275871748, P175=0.0002147828560355079, P176=0.00019707137908131945, P177=0.00027018224119029267, P178=0.0001337940840270561, P179=2.0715602112276786e-05),Row( P0=-642.2171014385854, P1=58.85727326485303, P2=-11.751148295048829, P3=5.837412706820615, P4=-13.77490148772146, P5=-13.148384366303555, P6=-6.6315615550871625, P7=-0.3328475374778728, P8=3.1332348257198683, P9=-3.4879004014336146, P10=-8.604991512842991, P11=-1.5378954952353525, P12=-3.929284140026442, P13=-4.996902356070564, P14=-5.004669731743322, P15=-5.765368018985394, P16=-2.0552406086402786, P17=-5.867900433159837, P18=-4.614464898123339, P19=-2.466543342354457, P20=-5.413881180787088, P21=-6.332451434908814, P22=-2.8351772149880157, P23=-2.264731116621605, P24=-7.863755674525298, P25=-0.10096556412469467, P26=-4.1563498414859925, P27=0.6816311735114535, P28=4.337276165214169, P29=6.475581409992115, P30=8.604731623828972, P31=9.435331364483302, P32=3.9252815049191225, P33=4.665739994399978, P34=-2.210191696246912, P35=-1.248013056275993, P36=-2.976587589206255, P37=-0.4000883862396857, P38=-0.9016364419369998, P39=2.0665814632720427, P40=0.7328461543593006, P41=0.7871253780168128, P42=0.7726459441369247, P43=0.7141237821269287, P44=0.6827909417747356, P45=0.6788934671844998, P46=0.6507532773972222, P47=0.6587466291419952, P48=0.7345472890577625, P49=0.7283017443398246, P50=0.6737269394326042, P51=0.6636664506073257, P52=0.00014345140525615443, P53=0.00011836383555209404, P54=6.481405179816357e-05, P55=1.9244519250164832e-05, P56=0.00010931790204803571, P57=0.0009939821595515842, P58=0.006243427671841344, P59=0.05691464905906698, P60=0.11018597031518924, P61=0.042189232359665756, P62=0.013870106165139728, P63=0.001954204970618943, P64=0.0004512943468194062, P65=0.0010284806024362589, P66=0.0035511182109033103, P67=0.01478865582402114, P68=0.08160252582491026, P69=0.10448559341784372, P70=0.034587527166285534, P71=0.02078897222143023, P72=0.01728128713824243, P73=0.006985368682443566, P74=0.0034145491826755017, P75=0.008650207661777034, P76=0.027975850337447223, P77=0.0771009572190447, P78=0.062267902157993014, P79=0.026776959955109574, P80=0.019931251901130723, P81=0.02645207870999272, P82=0.030769660578963208, P83=0.02943911557790104, P84=0.053642439250376386, P85=0.06382596911382249, P86=0.05834661504996996, P87=0.021492331477982622, P88=0.002240128569776955, P89=0.0012937405797857965, P90=0.005989586044807298, P91=0.023382664941821552, P92=0.029628733187654986, P93=0.014059967589582204, P94=0.002640455681960238, P95=0.0007516551043466903, P96=0.001442017894608347, P97=0.001393479121404135, P98=0.001905724901523772, P99=0.0011471277963488488, P100=0.00017966120290391197, P101=0.00017301788212011038, P102=0.00043340275798460704, P103=0.00045208956143840865, P104=0.000388328856754183, P105=0.0007351909848090374, P106=0.0009126902979029923, P107=0.00046222221367055893, P108=0.0006343914715786443, P109=0.0013007229893405422, P110=0.005192771858377384, P111=0.00616281717421108, P112=0.0018653448970563668, P113=0.0008074724113091864, P114=0.0007976061820753229, P115=0.002140459959299223, P116=0.002666510691843417, P117=0.0011460679696155135, P118=0.001003528813448075, P119=0.0020076197456758, P120=0.0016035485756450668, P121=0.0010318170481033735, P122=0.002780308060153342, P123=0.0008429400361889162, P124=0.00027353289259346593, P125=0.00021420283131854518, P126=0.0001836487636685763, P127=0.00022685584819339945, P128=0.00014305216412342282, P129=0.00022336909127320913, P130=0.00036909477733609153, P131=0.0007342267189685495, P132=0.0006518312996422711, P133=0.000500473591580824, P134=0.00038145249087399836, P135=0.0002449620775649123, P136=0.00015731697236605726, P137=6.286969889042569e-05, P138=6.270683392958407e-05, P139=5.958722412207193e-05, P140=8.84461698316326e-05, P141=0.00019234625814779934, P142=0.00041143869747566964, P143=0.0006412869624508395, P144=0.0004688775384601933, P145=0.0003245453568579377, P146=0.0003127798507718214, P147=0.00013031319217894047, P148=4.362811537751207e-05, P149=6.711810965688171e-05, P150=0.0001475489892615223, P151=0.0001791787945542669, P152=0.00021489184386999028, P153=0.0003455432804053151, P154=0.0003362510703142937, P155=0.00038770338799575806, P156=0.00023789397666325023, P157=0.00014942410322359628, P158=0.00011204642945702756, P159=0.00010515475801222709, P160=0.00010197643626926689, P161=8.173229814133727e-05, P162=5.594168861469699e-05, P163=3.347446740383694e-05, P164=2.5966849512699957e-05, P165=2.692292138748636e-05, P166=1.9203489763926045e-05, P167=3.826645019686936e-05, P168=6.951935792024394e-05, P169=6.41827209509076e-05, P170=4.5575158382793e-05, P171=3.7645563233083196e-05, P172=2.7655980004252552e-05, P173=1.73931967785021e-05, P174=1.9729139804912635e-05, P175=3.7424041266425254e-05, P176=6.31719029107883e-05, P177=7.265770336781612e-05, P178=3.957600273095441e-05, P179=4.768921996070783e-06),Row( P0=-664.321356694342, P1=61.75185072270062, P2=-14.408447470598306, P3=0.5597726438742237, P4=-15.042130011993944, P5=-15.577979660913533, P6=-13.109161916468492, P7=-4.865647132501809, P8=-5.806715364640887, P9=-4.895321064318527, P10=-13.116239499283127, P11=-4.92704711076252, P12=-2.926946343273464, P13=-7.865055616883508, P14=-2.471534942206584, P15=-5.157380606314254, P16=-0.15344967586406918, P17=-4.603553545555086, P18=-1.7412091779895746, P19=-0.21099404502388538, P20=-4.660762609607795, P21=-2.735753609784922, P22=-3.2906971274455703, P23=-1.5512276002166379, P24=-3.5606559451059585, P25=0.5634622713236145, P26=-5.318272359691904, P27=0.48533650643121884, P28=-1.9077569532907013, P29=-0.7946375569249815, P30=0.8993540689202314, P31=2.455946889769492, P32=0.47130953277778603, P33=2.0370673527699785, P34=-1.4259575066952281, P35=0.28380249845001815, P36=-2.514091205397821, P37=-0.20544257977648103, P38=-1.7121262760221247, P39=0.5004524512820128, P40=0.7119549561908176, P41=0.6490856388608571, P42=0.6403211555940354, P43=0.6549696392148058, P44=0.665209626385409, P45=0.6594672295587996, P46=0.6825718679570614, P47=0.6667593391624429, P48=0.6618263114228096, P49=0.6609918739661772, P50=0.6682007325328755, P51=0.6977796788276537, P52=1.8325168673239207e-07, P53=8.227867048380546e-07, P54=6.031704745655334e-06, P55=1.6860768530210714e-05, P56=6.532493317802385e-05, P57=0.0005085298945730071, P58=0.003908964198910826, P59=0.014422955872453602, P60=0.019503764331919127, P61=0.018390826415111163, P62=0.016506049103711954, P63=0.015024483889550841, P64=0.007755784386336598, P65=0.033396741095057116, P66=0.03856874475996145, P67=0.01635304825610291, P68=0.013199561358944983, P69=0.014656773570030071, P70=0.011226728139246191, P71=0.006982969142587808, P72=0.00454236719576405, P73=0.004223650151018162, P74=0.006987087371608125, P75=0.01250112893666772, P76=0.007426099413931093, P77=0.006095794855656122, P78=0.00928260727135823, P79=0.02407727199846907, P80=0.04878255376307269, P81=0.026239471415414856, P82=0.009101958688960212, P83=0.006314215596679217, P84=0.008920631837675025, P85=0.005421926399286229, P86=0.003540013285564977, P87=0.005032568973042653, P88=0.0023695580265560325, P89=0.0027749463117099984, P90=0.005918822607440933, P91=0.0032322932764082763, P92=0.001353705814040522, P93=0.0005637957552434756, P94=0.0004302748360634581, P95=0.00047225176558599133, P96=0.0004535773113363989, P97=0.0002215764863768624, P98=0.00018772648601132728, P99=0.00014067952273529534, P100=0.00026051181085610063, P101=0.00023631565847803337, P102=0.0004131173718867402, P103=0.0006232466177769366, P104=0.00039913722244678504, P105=0.0003871040886374686, P106=0.00017616355630211395, P107=7.028071575199004e-05, P108=0.00012651653871573244, P109=0.00013386118976922414, P110=0.00012358965005219514, P111=0.0001745991311547729, P112=0.00016311454081823098, P113=0.00020622104225875897, P114=0.00046549633176562027, P115=0.0002473281709169526, P116=0.00019448549854741341, P117=0.00015363546158386694, P118=0.00016189695591948758, P119=0.00030455871771352573, P120=0.0003654778933380398, P121=0.0001160660575176661, P122=0.00011930938725947603, P123=0.00012721579061895672, P124=4.832602163371929e-05, P125=5.4537829280054765e-05, P126=0.00011927468934919284, P127=3.8886411635090793e-05, P128=2.4376411219911033e-05, P129=3.081617903108954e-05, P130=1.728752520528373e-05, P131=2.8680514001864193e-05, P132=4.513765667897367e-05, P133=4.706728953458954e-05, P134=5.0512605687356485e-05, P135=7.440575587809475e-05, P136=0.00010554349015017096, P137=5.081523685932064e-05, P138=4.830012136445351e-05, P139=4.3961565384144634e-05, P140=2.2798405061437938e-05, P141=2.4754345875634157e-05, P142=3.679138495945644e-05, P143=4.314502185027274e-05, P144=6.95122716057334e-05, P145=0.00011647143937626743, P146=0.00022987101373875377, P147=8.27994854959501e-05, P148=1.999129212365292e-05, P149=1.914714609719881e-05, P150=2.433632719401971e-05, P151=4.105984253246287e-05, P152=2.8683419702829195e-05, P153=4.494030312045346e-05, P154=0.00010635496301523089, P155=0.00014307580420279918, P156=6.0119903298810694e-05, P157=3.1082725170835366e-05, P158=6.409317578514828e-05, P159=4.9755574415350734e-05, P160=3.8488575069785006e-05, P161=4.265007678466348e-05, P162=4.485400845158397e-05, P163=3.7623843865144946e-05, P164=4.853693405322004e-05, P165=3.5982507954323584e-05, P166=4.392850952820201e-05, P167=2.6361144999941067e-05, P168=9.302709817988714e-06, P169=8.84260783402551e-06, P170=6.543884532615009e-06, P171=5.396214205015085e-06, P172=3.9742485751720406e-06, P173=3.443345520138382e-06, P174=5.0412290367978584e-06, P175=6.611763667959114e-06, P176=7.006491469546939e-06, P177=4.687303286649762e-06, P178=1.634956945495497e-06, P179=2.4980282116430273e-07),Row( P0=-614.1132604102423, P1=59.36646223250322, P2=-6.839480403069157, P3=3.2683565086819515, P4=-13.321099045667639, P5=-14.582413913404674, P6=-3.7036213625100944, P7=1.1698544582648898, P8=4.563509802115921, P9=0.6818569745134375, P10=-8.925465926007409, P11=-2.7201148867258533, P12=-5.539208432803469, P13=-2.860500474097567, P14=-2.5497080588634904, P15=-4.921972571896107, P16=1.3197444478758824, P17=-4.330343062979714, P18=-6.366102336593386, P19=1.0893433064657587, P20=-4.2328799205471075, P21=-1.0917006517046421, P22=1.8453868623124796, P23=2.001397726958189, P24=0.42372858455463036, P25=4.036757685030285, P26=0.19755125236919888, P27=3.6578075343201437, P28=5.44034100218013, P29=4.570430894556644, P30=4.465319758633783, P31=4.651881708723541, P32=-0.7944249631853441, P33=0.610589619669218, P34=-4.313580527793671, P35=0.08940903688558029, P36=-0.9793290280143515, P37=-0.3028863392788881, P38=-2.0571134870005454, P39=-2.194201637740124, P40=0.6814113485787522, P41=0.7426725142916619, P42=0.7858055219619007, P43=0.7368033298835996, P44=0.6908459297193398, P45=0.6811125701187617, P46=0.6878040874919199, P47=0.6531298377679319, P48=0.6014351704337325, P49=0.6265819035593466, P50=0.6475928414072146, P51=0.6476522965056957, P52=0.00020260670527524785, P53=0.00020798457346156679, P54=0.00017843424014493835, P55=8.447910495196504e-05, P56=4.9376703254578116e-05, P57=0.00023497945190128144, P58=0.0021279002010631955, P59=0.024987550329382765, P60=0.10197405451550519, P61=0.09836554099646624, P62=0.12832198679557466, P63=0.11412649485466558, P64=0.020671802026324706, P65=0.0041188301849091466, P66=0.0028349941119029075, P67=0.004143236749028965, P68=0.02842878790772051, P69=0.08507064489840496, P70=0.07821140838647193, P71=0.03937346275285709, P72=0.028157128720383253, P73=0.025611321106078205, P74=0.04914516139163649, P75=0.04827543802899962, P76=0.018502712508675877, P77=0.027444110644755156, P78=0.06829395896800146, P79=0.07481799999206723, P80=0.0530343362039247, P81=0.025601383779436587, P82=0.02076741952531072, P83=0.01662652188109589, P84=0.024762835246770795, P85=0.04071686494354262, P86=0.11517293544847253, P87=0.19572187316031026, P88=0.09767415187477171, P89=0.029883138891883296, P90=0.005962008586643801, P91=0.009541404745472989, P92=0.018508505961936754, P93=0.039933000892722496, P94=0.10028957338706927, P95=0.10325902024570598, P96=0.023453718166582285, P97=0.0006743703241164686, P98=0.0010137077670776208, P99=0.002590243348221486, P100=0.0029822799766735164, P101=0.0011648296973802686, P102=0.001352935377034733, P103=0.000594407307339475, P104=0.0010924705639112854, P105=0.003616603175766086, P106=0.008489457733887285, P107=0.0053524161585826666, P108=0.004901958163230112, P109=0.004437786326907336, P110=0.0063902804445819346, P111=0.0086387160048302, P112=0.003940739952096847, P113=0.001189409614926175, P114=0.0019846360135970867, P115=0.007767288850626166, P116=0.003190901132171762, P117=0.0011433823938891215, P118=0.0023525386804662375, P119=0.0016863508902643166, P120=0.0005621137427241835, P121=0.0013554576497349998, P122=0.0007730725767875687, P123=0.0003685182341226224, P124=0.0003787497126820803, P125=0.00047283729109924084, P126=0.0003558349155363448, P127=0.00026511958026874643, P128=0.00019614974761618674, P129=0.0002770271031227732, P130=0.00038793126587658886, P131=0.0008200705801349484, P132=0.0005494042034342916, P133=0.0015903782064214183, P134=0.0007720622380319661, P135=0.000709307317404711, P136=0.00031359126560431415, P137=0.00010924193042356966, P138=0.00010113834430913319, P139=8.000901171813567e-05, P140=7.887808556851207e-05, P141=0.00012745809122780377, P142=0.0001721311869532211, P143=0.0005693502113772092, P144=0.0009397674688930988, P145=0.0011948225418369042, P146=0.0006907365258687743, P147=0.00022802455964412887, P148=6.360706368644635e-05, P149=3.184089963802014e-05, P150=4.134911227397291e-05, P151=5.1763667262667163e-05, P152=3.9770500625987504e-05, P153=7.325534514705968e-05, P154=0.00012422557398792975, P155=0.0001833289737247344, P156=6.251858192590041e-05, P157=4.753435134035638e-05, P158=2.6931457195373398e-05, P159=3.0647315529880645e-05, P160=6.693728362459886e-05, P161=6.54443634338528e-05, P162=4.1516732440772435e-05, P163=3.772117783272725e-05, P164=1.870296387218845e-05, P165=1.1886647819327923e-05, P166=8.123659797679439e-06, P167=8.024894888778736e-06, P168=1.1669340978870969e-05, P169=1.4448925055690473e-05, P170=1.4125981273056312e-05, P171=1.406840429207481e-05, P172=2.21097126833511e-05, P173=2.2256152542642277e-05, P174=2.3304853149194394e-05, P175=2.265323913776722e-05, P176=1.3464382223118157e-05, P177=1.5389646665055645e-05, P178=9.105089165564828e-06, P179=1.534261364049864e-06),Row( P0=-724.6999889786501, P1=68.30635040799602, P2=-8.079345025136723, P3=3.365851908404309, P4=-8.214287907936258, P5=-13.513396593979218, P6=-6.849857337451961, P7=-1.2415725732098915, P8=-2.5062862451912813, P9=-2.8604236119091717, P10=-8.760233416079682, P11=-6.585300741985418, P12=-2.2816017583015733, P13=-6.8712953855293355, P14=-4.821280900539005, P15=-5.647687410966698, P16=-2.598576964573522, P17=-4.592709687105523, P18=-6.8461044144705445, P19=-3.223526956166642, P20=-8.734726993486452, P21=-6.468036300698986, P22=-4.078552643376769, P23=-4.009553861770375, P24=-5.537125426739376, P25=-2.3692982361706907, P26=-4.693595052544325, P27=-3.7993909050358217, P28=-2.2664734874617958, P29=-3.816453571653512, P30=-2.909102149643689, P31=-1.4965717710850182, P32=-4.122513423905198, P33=0.1831845481906952, P34=-2.0910027946156062, P35=0.8760005178116702, P36=-1.2336412993113242, P37=2.530034104735286, P38=2.20531902039874, P39=2.7232438285004323, P40=0.715478599108393, P41=0.7168108878047759, P42=0.7362582753492759, P43=0.733861195242789, P44=0.779097402912676, P45=0.7336880633472413, P46=0.6979427861301074, P47=0.7281170580551304, P48=0.7674596041745486, P49=0.7591518224218253, P50=0.7408168242256403, P51=0.7550043042372906, P52=9.55335923230622e-08, P53=4.4035034937090423e-07, P54=8.844442923805048e-06, P55=3.128043441078941e-05, P56=0.0005090784828933789, P57=0.006874226308774216, P58=0.010390601602973523, P59=0.006848090458976015, P60=0.0053089776935642385, P61=0.013921699781157867, P62=0.016228988620327754, P63=0.002653918161434292, P64=0.0020417261118559358, P65=0.005632157345141059, P66=0.005452642091770661, P67=0.0026803163647493473, P68=0.0025797175786811187, P69=0.002797182254623153, P70=0.0033646611517172474, P71=0.005452903550814812, P72=0.01305442692948779, P73=0.011668291694731058, P74=0.0026799817040925504, P75=0.0005533962330916894, P76=0.0005539706327511667, P77=0.001314754906267533, P78=0.002213944505680263, P79=0.0032747102987576203, P80=0.003963804957060436, P81=0.0033461863727211, P82=0.005850981641960432, P83=0.016281655524756525, P84=0.016648276148433488, P85=0.004685643414353943, P86=0.003003888908259463, P87=0.0029884861852186653, P88=0.000953389836074162, P89=0.0004914491784581582, P90=0.0005986792296164311, P91=0.0023621163945149104, P92=0.005489363914403384, P93=0.0017826129736328794, P94=0.0011143328484580367, P95=0.0007235426817339898, P96=0.0007302643359768558, P97=0.0016326398172770485, P98=0.0017533314121816284, P99=0.00030743332855550254, P100=0.00010875348044179494, P101=5.842666400546575e-05, P102=4.306785741839009e-05, P103=9.724974011595945e-05, P104=0.00014786735602129842, P105=0.00016829630899458303, P106=0.0002552156373416745, P107=0.00024931374452026947, P108=0.0001364076959440106, P109=0.0002680494611163486, P110=0.0005655888244869659, P111=0.0002919479242033892, P112=0.00015357161977162592, P113=0.00019320886038145602, P114=5.9928870179450665e-05, P115=0.00014276836653593307, P116=0.0003548153371139454, P117=0.00013696434227450586, P118=0.00012562982132120664, P119=0.0001525982942298595, P120=0.00010684170827451558, P121=0.00015766716434663502, P122=8.338651491642889e-05, P123=0.00017513158718132483, P124=0.000130358641163289, P125=0.00015487106343459932, P126=0.00024870303996470567, P127=4.7280758828406586e-05, P128=3.0515217392308692e-05, P129=3.467105022596455e-05, P130=2.069254428749445e-05, P131=3.726493871114778e-05, P132=5.17600974052209e-05, P133=0.0001252114308269324, P134=9.389305330021166e-05, P135=0.00010337929171203785, P136=4.189022778085366e-05, P137=2.294714018647278e-05, P138=1.5749738453374754e-05, P139=7.718638236830754e-06, P140=7.967890833853783e-06, P141=9.315740152477976e-06, P142=1.560835706507834e-05, P143=2.3328543350898906e-05, P144=4.2098632725049454e-05, P145=5.983542665286744e-05, P146=2.533954028059578e-05, P147=9.22149779101334e-06, P148=2.309348706262225e-06, P149=1.3276572422127996e-06, P150=1.1713126214859594e-06, P151=2.4487550680948446e-06, P152=9.146381671347281e-06, P153=1.2398530196189235e-05, P154=9.113785619072982e-06, P155=1.1538911058962598e-05, P156=8.46539280523679e-06, P157=8.292536036431938e-06, P158=5.9521256996169825e-06, P159=1.0753462182223793e-05, P160=1.4969101100941936e-05, P161=1.1367377355104507e-05, P162=6.977224651434702e-06, P163=4.867686064747104e-06, P164=2.631845913757153e-06, P165=1.7404837912161036e-06, P166=1.140774539653296e-06, P167=1.704627463865866e-06, P168=2.354995628250606e-06, P169=1.790860548652468e-06, P170=1.3716001903043857e-06, P171=1.2712358438178843e-06, P172=1.4097421367177577e-06, P173=1.7400880478931355e-06, P174=1.6111924365684822e-06, P175=1.3967497448839171e-06, P176=2.09743201539698e-06, P177=2.8949581099957294e-06, P178=1.6008333553228434e-06, P179=1.7397242146896242e-07),Row( P0=-643.8158781648672, P1=65.97637425813917, P2=2.2394541629167843, P3=10.33083632911445, P4=-7.974702413410747, P5=-8.083786175612861, P6=-3.44468595183748, P7=3.0923764415697224, P8=5.281246994013687, P9=1.487169427452205, P10=-5.9807439626203385, P11=-0.429088801559647, P12=-3.3204171146543784, P13=-3.8934139500594296, P14=-3.68521914323438, P15=-5.022269601336307, P16=-0.3769881736114603, P17=-3.1836899646495964, P18=-5.275768009197115, P19=-0.6621813093686552, P20=-5.823164445202029, P21=-5.710550129902841, P22=-3.0300223409591824, P23=-2.4386333562943756, P24=-7.518616110178009, P25=-0.2159015196668157, P26=-2.657496390542411, P27=0.2620067536169494, P28=3.016067632254461, P29=2.8377242440148818, P30=0.6314316350922055, P31=2.1326361997506975, P32=-1.9777379119210559, P33=1.8728205026681668, P34=-0.2430521800924299, P35=3.168392304438408, P36=2.7566978217646048, P37=2.915607426671889, P38=0.34519532103998024, P39=0.9281883715032231, P40=0.7076408709334794, P41=0.7138504560935297, P42=0.7406276785141211, P43=0.6915262253184543, P44=0.6641555645857273, P45=0.6805810602989659, P46=0.6697612107430799, P47=0.6496845223259572, P48=0.7333431672939262, P49=0.7790463172638348, P50=0.762578265873932, P51=0.700804226715873, P52=0.0003074484323995005, P53=0.000636940479539548, P54=0.0006749806022460876, P55=0.00022904813352117353, P56=0.0005982320057863459, P57=0.00771671454899259, P58=0.03342122761673496, P59=0.06236111950153442, P60=0.12812589485522002, P61=0.07661004127229475, P62=0.0138639064612235, P63=0.008995711985393149, P64=0.013363350377969986, P65=0.024309344768751167, P66=0.03940705901469238, P67=0.038684790947468684, P68=0.03346984882210027, P69=0.05776170005650941, P70=0.06466150377803284, P71=0.024609266206816794, P72=0.006537858583925151, P73=0.00874056964972922, P74=0.010860780376941177, P75=0.014278102366572797, P76=0.022774279029035255, P77=0.0664032845311006, P78=0.1176602742691723, P79=0.11028008196604931, P80=0.05574994831024403, P81=0.02865490138664324, P82=0.011895055123062064, P83=0.010222990678593733, P84=0.01956707021106025, P85=0.018485039020123443, P86=0.01712514859893558, P87=0.036125184411177465, P88=0.06638288307850293, P89=0.050502600005842516, P90=0.025853524891136865, P91=0.0029774138583487726, P92=0.0018369901843853906, P93=0.0036398878408899836, P94=0.014607137043694072, P95=0.014224610626805134, P96=0.009634316701442954, P97=0.0019804256372261773, P98=0.0006452962940319436, P99=0.0006372302105823566, P100=0.0008722769385697146, P101=0.0007631454649756964, P102=0.0008957732820831394, P103=0.00024261762752815489, P104=0.00025070366059153835, P105=0.00140411787285193, P106=0.0015716483500302367, P107=0.0006024174226072159, P108=0.0012758648210385126, P109=0.0016982920125993655, P110=0.004848223286603503, P111=0.004715468593801529, P112=0.0010126962953847566, P113=0.001229960540343792, P114=0.000852847922446628, P115=0.002286857348094757, P116=0.0011342523953974098, P117=0.00055990817386945, P118=0.000957887876217497, P119=0.0011822424145846854, P120=0.001598311996180473, P121=0.0010753608732609991, P122=0.0011225023905759334, P123=0.0005612710110561555, P124=0.0005690600006477959, P125=0.0003893538555431531, P126=0.000294668142144751, P127=0.0003792461889562782, P128=0.0002547432398260971, P129=0.0002912317145098297, P130=0.0003800286846037038, P131=0.0007229155812794553, P132=0.0005755950255368038, P133=0.0007027339734646146, P134=0.0005507239623872518, P135=0.00028714071014339056, P136=0.00016049500462434846, P137=5.749389450859802e-05, P138=4.6777779856718776e-05, P139=5.851544988554425e-05, P140=6.810123457020415e-05, P141=0.00014588261790826994, P142=0.0002562353024600289, P143=0.0005748880673667884, P144=0.0009878620117441531, P145=0.0015265794287683504, P146=0.0012964106623286446, P147=0.00024297284352836298, P148=0.00027866286005666184, P149=0.00017044265041622932, P150=0.0006351778767698871, P151=0.0003066016604188408, P152=0.0001253903584492616, P153=0.00025468063157109227, P154=0.0002766121592636782, P155=0.00021040977107550165, P156=0.0001997776244149053, P157=0.0001324075589098371, P158=0.00014157605610408026, P159=9.055437527622798e-05, P160=7.834396656733959e-05, P161=5.927145661446514e-05, P162=7.68197674959482e-05, P163=3.893299785937375e-05, P164=3.329338673364116e-05, P165=2.2807364743117132e-05, P166=2.7705639532407548e-05, P167=2.093592670739467e-05, P168=3.036445208482595e-05, P169=4.4881773227247956e-05, P170=6.154436854627906e-05, P171=4.551794903421102e-05, P172=2.3147517758798075e-05, P173=1.641633146392845e-05, P174=1.7309186784367392e-05, P175=3.775597965999242e-05, P176=5.02068780380698e-05, P177=4.345184742987872e-05, P178=2.0076296855790155e-05, P179=2.155933980865664e-06),Row( P0=-440.20694909642407, P1=34.7375102676821, P2=-30.89708183587405, P3=-5.900869791769035, P4=-13.792664296507686, P5=-22.739521209309288, P6=-19.838891292977213, P7=-4.195350240687124, P8=-2.337689837832492, P9=-5.4966316332629015, P10=-11.344953560375552, P11=-1.5909616661923571, P12=1.3235822515791247, P13=1.7788583024473392, P14=10.384522871797897, P15=13.19585677636391, P16=11.015310949946093, P17=5.017688261912296, P18=-1.4814944906271912, P19=6.6618724470024135, P20=-2.2583870555219994, P21=-1.8096942863059686, P22=1.956655031091857, P23=-1.3075027699454809, P24=-3.675346498134397, P25=1.5306931816331795, P26=0.7318381309316783, P27=1.0475011874098203, P28=-0.16609819044510693, P29=0.3953210620667587, P30=-0.8023828846795276, P31=1.890020696409594, P32=-3.6268822971337693, P33=-1.602714774342925, P34=-2.016046828748926, P35=1.1363658498325784, P36=0.020097836721840952, P37=-3.3077853054075312, P38=-1.7089344704425893, P39=2.435397347101033, P40=0.6105950210510734, P41=0.5424820896950622, P42=0.598174424829938, P43=0.618011906103365, P44=0.5337304853081646, P45=0.5429421350484466, P46=0.6302269369352517, P47=0.5820541874991634, P48=0.6016424831127505, P49=0.6720407959892716, P50=0.6360673523620902, P51=0.622075282736148, P52=0.0017188510137690772, P53=0.001981215779612828, P54=0.0007451783104218729, P55=0.0006419250099986763, P56=0.0006113864802488538, P57=0.00022726657190860706, P58=0.00012819765218652801, P59=0.00012719414271160107, P60=0.00020491083772943434, P61=0.000897340352834403, P62=0.01966203845184259, P63=0.057785170126696636, P64=0.07119552390160791, P65=0.32989286641619886, P66=0.6431277629141106, P67=0.6660159606364437, P68=0.5136627082645193, P69=1.1380919468829753, P70=2.5903453913358057, P71=1.885455507780081, P72=0.4955807510451777, P73=0.07855385963894854, P74=0.010226053468391968, P75=0.008212972429502068, P76=0.010202822000350472, P77=0.03459654797801944, P78=0.08935275787685301, P79=0.32945696257461515, P80=0.5494623115884569, P81=0.29448140940295325, P82=0.25376455849677915, P83=0.3760076899814935, P84=0.7284775092149605, P85=0.43618191337702544, P86=0.3179837614195959, P87=0.535307984908847, P88=0.8050767275988646, P89=1.2595982780491781, P90=0.9051184154369404, P91=0.3941363081987406, P92=0.2465640130611544, P93=0.04851328609678231, P94=0.08868013405809692, P95=0.13760558724486605, P96=0.09205773267965153, P97=0.07074500137142646, P98=0.10965837679171829, P99=0.13714817385837116, P100=0.061356367246559906, P101=0.07204817200628721, P102=0.14850758479309822, P103=0.08051291914109444, P104=0.04165967094443786, P105=0.033229584505347104, P106=0.02573897108492909, P107=0.014816117324976872, P108=0.06336420946281063, P109=0.2774561327012024, P110=2.5705055234947385, P111=3.8323992809676084, P112=0.12111717255682884, P113=0.014271387630960305, P114=0.01626871738547545, P115=0.029965291040686057, P116=0.11143450187275819, P117=0.09456412672731128, P118=0.05743915421166552, P119=0.034832534067090484, P120=0.026786674426382485, P121=0.03545726689018549, P122=0.03938317390317728, P123=0.021695725750394492, P124=0.016494425309580483, P125=0.011122886372097223, P126=0.005560701901779424, P127=0.005509771850682243, P128=0.005781071010260547, P129=0.005655610198306443, P130=0.004773905777007148, P131=0.013765448434105374, P132=0.024066929346736654, P133=0.01947908043312369, P134=0.008987179366485281, P135=0.006397902337129494, P136=0.006626086869753233, P137=0.003359718872471049, P138=0.002222011971555957, P139=0.002631816087420235, P140=0.003484263907372739, P141=0.0044422806133538745, P142=0.0062115383864079455, P143=0.00658697996724003, P144=0.009412526749971567, P145=0.007921625203224212, P146=0.003954464008100468, P147=0.0014889310400996428, P148=0.0010384812546133096, P149=0.0016123981907736598, P150=0.0009747941389452064, P151=0.0009872161016514747, P152=0.0010252022414546295, P153=0.0019095241448926822, P154=0.0005976317231425924, P155=0.0005346907617815144, P156=0.0008428610003210216, P157=0.000863208414241011, P158=0.0016555751394637858, P159=0.001675113276567358, P160=0.002642138178402145, P161=0.000816310563575633, P162=0.0005905602739111957, P163=0.000631172187497035, P164=0.0002548734875337696, P165=0.0001871101294316148, P166=0.00024025215639693657, P167=0.00027054149870480766, P168=0.00020878739084214962, P169=0.0002579993484502322, P170=0.00023254055004171684, P171=0.0002384815329517447, P172=0.0003118424302034104, P173=0.00048452153707815725, P174=0.0002669040585482763, P175=0.00022674135938799422, P176=0.00030917395354066357, P177=0.0003051973686191317, P178=0.00014386559787262388, P179=1.905040778685434e-05),Row( P0=-717.8181369842418, P1=69.7703740518767, P2=-8.818189419661149, P3=5.9564958363699105, P4=-8.617849309686772, P5=-10.707717933655811, P6=-6.456994852048925, P7=-2.3166653578886525, P8=0.8056686030320996, P9=-2.6963075417198175, P10=-8.490856383856487, P11=-6.730387856296267, P12=-3.274617097630116, P13=-5.845120114913609, P14=-3.8531185906261256, P15=-6.061480144658579, P16=-1.864851064359522, P17=-6.107546352294131, P18=-5.962752335157873, P19=-2.1321975635526527, P20=-7.952601809163682, P21=-5.103935998380592, P22=-3.4949958540692165, P23=-5.044992629019688, P24=-8.727240192568411, P25=-3.814119544549547, P26=-7.154406362602992, P27=-6.213398987905222, P28=-5.6705579479131245, P29=-3.8869619023676325, P30=-3.244219488167981, P31=0.32026507142384975, P32=-1.5875162935090679, P33=3.2406149743162387, P34=0.5226958822575134, P35=3.9058224776931447, P36=0.5233288569145131, P37=2.984446749452523, P38=0.6942192066628706, P39=0.5383154490502806, P40=0.7444836578006773, P41=0.6775250571098371, P42=0.6989176443731983, P43=0.7175181460517958, P44=0.7176712676375516, P45=0.6938030685852126, P46=0.6770052577112663, P47=0.6711523457006907, P48=0.7053909072867293, P49=0.7398526781523651, P50=0.760829533885597, P51=0.7502826563906655, P52=1.7560694701655143e-07, P53=1.654456266098759e-06, P54=1.1866286212599776e-05, P55=3.817299264854135e-05, P56=0.00045960865588232693, P57=0.00674479058875313, P58=0.029541253794394297, P59=0.04315118822010309, P60=0.013776285763657687, P61=0.0013102543098943364, P62=0.0005640994178472291, P63=0.0006972005658083617, P64=0.0018251988821450667, P65=0.013257967255831608, P66=0.03138371890523276, P67=0.029526121335398184, P68=0.012815781668032369, P69=0.0028633699793881827, P70=0.0010281589965009299, P71=0.0012218258558955109, P72=0.0024163883070027724, P73=0.004433628218273052, P74=0.0071371141432231095, P75=0.009885858964869584, P76=0.005716634849104267, P77=0.0038730175061032586, P78=0.002420435054542871, P79=0.003973298962197775, P80=0.007255687609388198, P81=0.007221072236938935, P82=0.006588864177824947, P83=0.00887872301194978, P84=0.01670741309258913, P85=0.015350318845032568, P86=0.0058080952627802785, P87=0.011107803713652971, P88=0.01980872965086001, P89=0.012147217578396874, P90=0.010113298591498251, P91=0.012598069627112529, P92=0.011363211455585035, P93=0.010466457557852527, P94=0.00919364760103469, P95=0.0036860068723889687, P96=0.001064086915291585, P97=0.0005548449556144503, P98=0.0005803600365099327, P99=0.0003655800377063835, P100=0.000336418444101595, P101=0.00027767964938954373, P102=0.0005273176021942802, P103=0.00024570856986919786, P104=0.00015026107198616126, P105=0.0003187945760805981, P106=0.0004441701422195649, P107=0.00020233956900234107, P108=0.000516799071889623, P109=0.0015994311826540812, P110=0.002508977525865686, P111=0.002806349694336628, P112=0.0017042539801325352, P113=0.0005813461662117966, P114=0.000649124799112449, P115=0.0006468358348853164, P116=0.000515156913637024, P117=0.0001961125716051861, P118=0.0002691758501050571, P119=0.00025237667170326534, P120=0.0001613377545630851, P121=0.00029407017095760507, P122=0.0003087625803005545, P123=6.306790851820236e-05, P124=8.66581871967188e-05, P125=3.99611890090278e-05, P126=4.87150013882161e-05, P127=1.6621427234283496e-05, P128=3.0120288992438923e-05, P129=2.9676976575236234e-05, P130=3.097241220597314e-05, P131=5.203976047199165e-05, P132=7.368052321199852e-05, P133=0.00011185196284111097, P134=0.000175540524959417, P135=0.00017333077118751293, P136=0.00013869306063097665, P137=3.365729727755783e-05, P138=2.0865559975991887e-05, P139=1.4561142126039914e-05, P140=1.824029357022652e-05, P141=2.5563048439005274e-05, P142=3.5019392102548345e-05, P143=4.5615644355026106e-05, P144=5.7265558976426386e-05, P145=5.392332192628307e-05, P146=9.066736640935028e-05, P147=4.37001871765486e-05, P148=1.696180538226775e-05, P149=1.2864141700836426e-05, P150=2.119667384859695e-05, P151=4.972758090361875e-05, P152=0.0001224713182622898, P153=0.0001349647571595159, P154=0.00022274213447533113, P155=0.00018380588612673405, P156=5.633477881892803e-05, P157=4.4390529337906556e-05, P158=3.541712631841964e-05, P159=3.070170117485408e-05, P160=3.9820792467579685e-05, P161=3.842526343007562e-05, P162=1.8845151991595193e-05, P163=1.5719921830419534e-05, P164=1.0599602291733617e-05, P165=7.358605683481186e-06, P166=6.192862514476974e-06, P167=1.5103896160550848e-05, P168=1.1344677155726542e-05, P169=5.231121417206001e-06, P170=1.5828484295081246e-06, P171=1.2063545473767645e-06, P172=1.6659672329036297e-06, P173=3.2599729149068173e-06, P174=7.2730937163688205e-06, P175=1.0857846008933875e-05, P176=1.2001349406872819e-05, P177=1.1515416163232202e-05, P178=7.138188213270046e-06, P179=7.295668320674716e-07),Row( P0=-646.3023969293564, P1=61.92421262413394, P2=-13.978070520828268, P3=6.538177927174142, P4=-14.617280853328085, P5=-14.17456731998292, P6=-6.318633289280374, P7=-0.8028787026932309, P8=-2.3877665149861897, P9=-0.6856772953419583, P10=-12.149867259790744, P11=-3.0696649942913528, P12=-4.053057351149412, P13=-7.889726335780336, P14=-0.5191499691163669, P15=-7.566874164319083, P16=-1.896013539382844, P17=-3.9451422752152827, P18=-7.352130845551795, P19=0.7346850825212193, P20=-7.438187101545041, P21=-2.7004458068068105, P22=1.933094953358606, P23=0.49318408827317484, P24=-1.258053884555861, P25=3.545776568172233, P26=-1.7027310790911798, P27=1.7072728733751887, P28=3.326048251854826, P29=0.3483760710516347, P30=3.4660818234339685, P31=3.7971019955122425, P32=-0.2142935467818335, P33=-0.06295095853011388, P34=-4.084644142931624, P35=-1.5241712689185498, P36=-2.5163576395124077, P37=-1.5809404248182748, P38=-0.13935566018257617, P39=-2.052227243513143, P40=0.7547515400068532, P41=0.7761204540135163, P42=0.7586213527903274, P43=0.74232739412139, P44=0.7330166974228951, P45=0.7360180873464309, P46=0.7180132607168789, P47=0.6901911128521053, P48=0.6846741781934946, P49=0.6943851590570073, P50=0.6827627096505211, P51=0.694608727537392, P52=0.0007368374135101471, P53=0.0017054091454313037, P54=0.002568925399260949, P55=0.0012187450303200319, P56=0.00021542193233141784, P57=0.00030158028530764906, P58=0.0017961988822603261, P59=0.014240358454828987, P60=0.04072893786132811, P61=0.06697633425025278, P62=0.08305504216273285, P63=0.04490296124010955, P64=0.012903949634361282, P65=0.0024255400713724114, P66=0.0029830511991272898, P67=0.007379619069575128, P68=0.027568364311298256, P69=0.03903054106211355, P70=0.04080102466825191, P71=0.024126336001021122, P72=0.018446829249698676, P73=0.02681204007396345, P74=0.03150327663584427, P75=0.034526025298401965, P76=0.05964890706271093, P77=0.06560093575367491, P78=0.041284559640111136, P79=0.025858699600504808, P80=0.019160510387429178, P81=0.01436197652349169, P82=0.018680762756998225, P83=0.04038302846650405, P84=0.06915675234166926, P85=0.09039502793518527, P86=0.04450182716941517, P87=0.023033485967368886, P88=0.0167482615872772, P89=0.006331175631024578, P90=0.004355478358818788, P91=0.015407045271766997, P92=0.07124136856075838, P93=0.025764273372941602, P94=0.05130515801988027, P95=0.06800216528557862, P96=0.008991502594409471, P97=0.002130246173979795, P98=0.0032675130088406635, P99=0.0029250324205367596, P100=0.0011727942473851367, P101=0.0007071614285807606, P102=0.0005103036758680853, P103=0.0005029806392015848, P104=0.0009761183153611267, P105=0.0018158690964608843, P106=0.00242096720783831, P107=0.004128335186426416, P108=0.0018838172481958867, P109=0.0013956466487020887, P110=0.004811410575731301, P111=0.0064506389388205455, P112=0.007975756310167254, P113=0.0028472727838456664, P114=0.001196809953689002, P115=0.0008868123866964178, P116=0.004730870617602915, P117=0.00378569618274844, P118=0.0007067160260886904, P119=0.0021243686544826436, P120=0.0010659134791933267, P121=0.0019855885330707012, P122=0.002221347407213592, P123=0.0005819722114856999, P124=0.0006596931409961554, P125=0.001857854779863671, P126=0.0012270223277909378, P127=0.0005914684092656488, P128=0.00026718370265306176, P129=0.00023612671115913985, P130=0.0004239674174091712, P131=0.000693950248829013, P132=0.0006100018981250534, P133=0.0012186927868331678, P134=0.0006174955224337964, P135=0.0007171333556853432, P136=0.0003624907204726881, P137=0.00011241305534309196, P138=0.00011021495761782855, P139=0.00010841394728405103, P140=0.00013419727871527234, P141=0.00015794192323197517, P142=0.0003306651429849304, P143=0.0008087827082556069, P144=0.0011686891600200982, P145=0.001404539846649238, P146=0.00046393086798039544, P147=0.00017183833882615464, P148=3.838509647398036e-05, P149=8.245858022594028e-06, P150=8.768530688759362e-06, P151=1.404161320340498e-05, P152=9.928731571644608e-06, P153=1.1960294285405153e-05, P154=1.8503632909915427e-05, P155=2.3765865587244635e-05, P156=1.734241192511798e-05, P157=1.8646108105395123e-05, P158=9.776043749944403e-06, P159=2.000000183680136e-05, P160=5.613602845731492e-05, P161=4.314864391463041e-05, P162=2.983685854838153e-05, P163=1.8311763522066367e-05, P164=1.5660354059358657e-05, P165=1.3191654445541953e-05, P166=4.719764883596806e-06, P167=7.616261358376506e-06, P168=7.941222353067012e-06, P169=6.260127953869527e-06, P170=3.7301380641396724e-06, P171=1.8429382590299546e-06, P172=3.3195925543038665e-06, P173=5.548607626073455e-06, P174=3.2150596968743204e-06, P175=2.097172585567261e-06, P176=1.2615404419906056e-06, P177=1.1451674992625518e-06, P178=9.796426196033577e-07, P179=1.985561280541816e-07),Row( P0=-566.7130046206387, P1=43.9709431848102, P2=-22.144305580563373, P3=-4.830331163736354, P4=-20.343393620356508, P5=-17.71949233171873, P6=-14.10130754489726, P7=-4.962422062007217, P8=-4.807652084988366, P9=-4.499049413971657, P10=-15.758271594213982, P11=-8.540785995801581, P12=-5.518442348363112, P13=-9.494466996497685, P14=-4.4529131348527, P15=-8.875256157290789, P16=-6.377626702805424, P17=-0.7722644901905386, P18=-4.888313806349284, P19=5.114896752694636, P20=0.5448800806502295, P21=3.3159988504157583, P22=5.966107456632895, P23=3.534253537301132, P24=-0.9547494315530524, P25=2.8374391662708804, P26=-1.593806805792223, P27=-0.47354093807744463, P28=0.4929794502703499, P29=-2.4617599479627152, P30=-3.437831170987231, P31=-0.825873045136114, P32=-5.4882578500661126, P33=-1.5938971846317402, P34=-4.268560868686257, P35=-2.888860665882364, P36=-3.053064153539803, P37=-2.150367395785419, P38=-2.065361132530973, P39=-1.3308176623027772, P40=0.6205855637589293, P41=0.6436271148478523, P42=0.6606950874569547, P43=0.660202774223849, P44=0.6789280045965556, P45=0.6730983083012566, P46=0.6886396467276221, P47=0.7009260098282715, P48=0.6707643034198074, P49=0.7021763489354341, P50=0.6860686973953853, P51=0.6268994973684173, P52=3.321354898530845e-07, P53=2.5645398339368444e-06, P54=2.312020695571417e-05, P55=4.4768094795506774e-05, P56=0.00010635002268650191, P57=0.00040440079004618805, P58=0.0006593272456958571, P59=0.00197618877343916, P60=0.011965474868016624, P61=0.05334270353871548, P62=0.14404698840723285, P63=0.19278200862202516, P64=0.14462212016659487, P65=0.23612065728358844, P66=0.1417534767398744, P67=0.02290319151882987, P68=0.006865248535626363, P69=0.00475560315880803, P70=0.007142892419420207, P71=0.017393172316829165, P72=0.022847611527372025, P73=0.04304061725404358, P74=0.10671856032260428, P75=0.1301259919573497, P76=0.08287288991794112, P77=0.08288271157792221, P78=0.13399101941749747, P79=0.29462613837868, P80=0.22851054261443918, P81=0.13998073247347148, P82=0.09200762081269705, P83=0.058051363435880764, P84=0.09840762229161842, P85=0.1470126002302057, P86=0.2354324205084331, P87=0.16748511718750494, P88=0.1367946927165677, P89=0.10211443161174093, P90=0.10180580780762001, P91=0.1456771165172295, P92=0.22142040215987904, P93=0.1982699898274384, P94=0.19502633390187094, P95=0.1068747073800335, P96=0.048651267518826785, P97=0.02494949583117508, P98=0.024212884529581966, P99=0.03841799320229655, P100=0.015247830883539133, P101=0.0037234402107038174, P102=0.004023531182726674, P103=0.005924553950716307, P104=0.007318075048569711, P105=0.015303299380013988, P106=0.0243449237646377, P107=0.03574550730750047, P108=0.010284472954102463, P109=0.031214234984641338, P110=0.027206095896470515, P111=0.028520207164729156, P112=0.02501612155188193, P113=0.01644016007710753, P114=0.007447785387595963, P115=0.021234006085894052, P116=0.024697019121379844, P117=0.004864068317416393, P118=0.013729892648623327, P119=0.01935546609558183, P120=0.006954986883710541, P121=0.006833823883974938, P122=0.006556672715656892, P123=0.0056697807537819675, P124=0.002956633710486222, P125=0.004528746649295046, P126=0.004954199195617879, P127=0.0015325152938694636, P128=0.0007281921396857141, P129=0.0013302487969576362, P130=0.0011801613323682589, P131=0.0019378141782018818, P132=0.004251769059916528, P133=0.0024718778735493374, P134=0.0021525636081492702, P135=0.001977424763733383, P136=0.0012764526470253207, P137=0.0011380673509377775, P138=0.0005321483610268635, P139=0.0006209684317901566, P140=0.0009463349565411159, P141=0.0018373227430986754, P142=0.002315764183816451, P143=0.0031473042241831756, P144=0.004584735059290043, P145=0.004340405202754399, P146=0.0015352824886992306, P147=0.0001449140990451922, P148=0.00010563341648744918, P149=0.00010748359289021478, P150=0.000994867298775655, P151=0.0015817253431143602, P152=0.0016191018055258146, P153=0.0012708663252893348, P154=0.0008468319685412338, P155=0.0007885044391115472, P156=0.0009599582055342144, P157=0.0007733577463850995, P158=0.0005206926515118824, P159=0.0005692507884830151, P160=0.0006154779721176328, P161=0.0005536210625009915, P162=0.0006028876461227266, P163=0.0006361648294149832, P164=0.0006658502591184907, P165=0.0006259822382443473, P166=0.0005105666348915747, P167=0.0003777998219139744, P168=0.0004088418318628526, P169=0.0004722586642762086, P170=0.0005641218300087465, P171=0.0004530732309230648, P172=0.00037566000797701623, P173=0.00037121708509752643, P174=0.0003822053269015563, P175=0.00037198243000355493, P176=0.0005039712762074536, P177=0.00036726907124343133, P178=0.00010372509332539327, P179=1.0124275729263117e-05),Row( P0=-519.6817687425793, P1=27.860488944099252, P2=-24.017102654191977, P3=-13.667581088338427, P4=-25.349081046794804, P5=-17.09757661264741, P6=-15.243906035437353, P7=-2.0655349357402017, P8=-8.05105694878233, P9=-11.508843223489396, P10=-8.30461247385713, P11=-4.375896338413792, P12=-4.075600066833029, P13=-4.74594440361703, P14=-3.1564975164547424, P15=0.4348743333944651, P16=3.3538131495099477, P17=-1.313896290087613, P18=-0.3607445012121827, P19=5.194254462619514, P20=-1.8788971553975407, P21=2.6643612545185738, P22=-3.785646486231796, P23=-3.157379554616568, P24=-2.1264554829636504, P25=1.1152078717034402, P26=1.0626339413950585, P27=-0.6592894494599675, P28=-1.6430213035825683, P29=-1.6907374929996397, P30=-3.063579879429853, P31=0.3624979732192601, P32=-2.4478807953886674, P33=-0.7152130920491675, P34=-3.012461897063014, P35=-0.1767223371784563, P36=-2.8005524558936528, P37=-0.6010810511784953, P38=-1.0165520382466278, P39=1.764549553024306, P40=0.6543787918574063, P41=0.5850353483880456, P42=0.5925916815934947, P43=0.6597588851104325, P44=0.6678041227225898, P45=0.6247464997752056, P46=0.6555363088491619, P47=0.6386392375409551, P48=0.6610633433546363, P49=0.6813667883027155, P50=0.6733045126973801, P51=0.6696769036542713, P52=1.9060731396984095e-07, P53=9.729021026753155e-07, P54=1.7269411775799117e-05, P55=4.7145715588502736e-05, P56=6.37045554936137e-05, P57=0.00018394156584724947, P58=0.0003717168184852649, P59=0.0007349456380340547, P60=0.00232643433410131, P61=0.006413509237801532, P62=0.015680216728119922, P63=0.04125586914694412, P64=0.11108147964499153, P65=0.20976655738295763, P66=0.27200059206135097, P67=0.27440939895889543, P68=0.16867591048014022, P69=0.2545399076670434, P70=0.3805670672929862, P71=0.34332776627659567, P72=0.17373138578997163, P73=0.06922964323916016, P74=0.057096194943165646, P75=0.07719485906975757, P76=0.0852914884056947, P77=0.13005541347416075, P78=0.26974929007082377, P79=0.2866994866008009, P80=0.35844265492493665, P81=0.4391268378200326, P82=0.7034466311808093, P83=1.0592878328157367, P84=0.47604224883017154, P85=0.0628685984378557, P86=0.02778619468962063, P87=0.09223992112804069, P88=0.15656913062380992, P89=0.126078131199231, P90=0.5482677397553779, P91=0.4845562878426762, P92=0.10029965130626987, P93=0.024735577034270626, P94=0.03424461608431705, P95=0.05738164951581109, P96=0.053156356485738736, P97=0.07676968588316131, P98=0.0531842649294674, P99=0.050775707620180736, P100=0.10176634867919236, P101=0.06468362623920515, P102=0.05568373642135609, P103=0.04019288982756715, P104=0.01467043950342343, P105=0.008911902131800686, P106=0.023717687320788765, P107=0.006280361825879451, P108=0.003175470209288024, P109=0.038116520579952566, P110=0.03692116769008475, P111=0.031180927840246075, P112=0.008279808093564371, P113=0.0013101015924883047, P114=0.001203844092966949, P115=0.00333039226993787, P116=0.01862734260838845, P117=0.015419789907929686, P118=0.0029723975908818347, P119=0.002233345830575978, P120=0.006891977324928306, P121=0.015110512890272399, P122=0.007312898038913217, P123=0.0024137783039108293, P124=0.0020245269692313393, P125=0.003485886582874229, P126=0.0027293286658766516, P127=0.0018447439325893357, P128=0.0019507740222900393, P129=0.0012402200783382924, P130=0.002016038320707768, P131=0.0012353176917301236, P132=0.0015325389991339537, P133=0.0025820268811170832, P134=0.0015644920694560262, P135=0.001309779483932018, P136=0.001028953246715324, P137=0.000811963611057996, P138=0.0008198914251718138, P139=0.00047478111890804153, P140=0.00031335168260327736, P141=0.00020054359678229298, P142=0.0004970248268073093, P143=0.00048615434038628267, P144=0.0007593555263488839, P145=0.001144417490297606, P146=0.0013400926735892198, P147=0.0016803566772058888, P148=0.0017225454552819957, P149=0.0013037176679644815, P150=0.0010411101562686712, P151=0.0011428400651389485, P152=0.0006979219944686874, P153=0.0006706833620724037, P154=0.0005187446860198594, P155=0.0003451859276910855, P156=0.00039284514908353893, P157=0.00046920841204273883, P158=0.0007203395096070505, P159=0.0009067329214318698, P160=0.0011094549287644306, P161=0.0013336653102785552, P162=0.0016430977984701353, P163=0.0011726133486366043, P164=0.0006653460765068923, P165=0.0005104326105950423, P166=0.0006790330608196232, P167=0.0008231287221448488, P168=0.0007635102461127926, P169=0.0009730307366354481, P170=0.0005753980206773039, P171=0.00043761819814111503, P172=0.0003525072130671572, P173=0.0004888865238948387, P174=0.00032982769003666077, P175=0.0003305520020497267, P176=0.0003317464290779685, P177=0.00022920235488289417, P178=8.373179262829013e-05, P179=8.289784780850547e-06),Row( P0=-712.3571670630948, P1=67.38790223037259, P2=-11.342278271742682, P3=3.2031810583226257, P4=-10.518619032239824, P5=-12.032912244209543, P6=-6.85181031330638, P7=-1.2610142111146656, P8=-2.017776588110341, P9=-3.9058193419716645, P10=-8.38970749251244, P11=-6.081746790185855, P12=-0.8320618930670316, P13=-6.139452916165877, P14=-5.7546805686130975, P15=-5.7345594658924695, P16=-2.6356250598775675, P17=-6.83126170416446, P18=-5.897876620931904, P19=-4.524199589365247, P20=-7.759121857899743, P21=-6.126984372299682, P22=-4.152964941956825, P23=-4.410527291030638, P24=-8.42240453446725, P25=-2.4505202664122177, P26=-7.074955946902999, P27=-4.0937816188983485, P28=-1.173385663405759, P29=-0.9029717022617219, P30=-0.1413567192502722, P31=0.8541342122750217, P32=-2.3341464382381862, P33=1.7090532427860987, P34=-0.5323054978482881, P35=1.9007297634974816, P36=0.41617561811649095, P37=3.3265239514184617, P38=1.8506509060411813, P39=2.51829288114203, P40=0.7071678863912572, P41=0.6924504272464257, P42=0.7207261349390296, P43=0.718677858706343, P44=0.7017732950574835, P45=0.740858658945725, P46=0.7192503502379658, P47=0.7111530940152725, P48=0.7312012425143876, P49=0.776454269011584, P50=0.7575318138455686, P51=0.7156851234062599, P52=2.1408400484231486e-07, P53=6.661416616530866e-07, P54=7.309824247605456e-06, P55=2.612562767588805e-05, P56=0.0004792505087964274, P57=0.005506491464112944, P58=0.011020804652858556, P59=0.014532116045250924, P60=0.015216131980014511, P61=0.0037032860835379084, P62=0.00033272095918342854, P63=0.0002382277632772019, P64=0.0007697404786587023, P65=0.0033616474196788006, P66=0.005702032687900562, P67=0.00523489753931125, P68=0.006339552833791649, P69=0.01078549804989682, P70=0.005019053333958831, P71=0.0010989979561266895, P72=0.001273691842254453, P73=0.0015057973200260613, P74=0.0015742051829342352, P75=0.0014839605951777375, P76=0.0019411096403307072, P77=0.0067350029777813415, P78=0.016336559053006565, P79=0.011380084173734734, P80=0.0019558379781104062, P81=0.0006115207498720145, P82=0.0006863345652695973, P83=0.0011016484816482632, P84=0.001681369876934988, P85=0.001987535051979127, P86=0.0023785080145232364, P87=0.0018034982112193824, P88=0.0010076740563629543, P89=0.0005134143952547544, P90=0.0006406794478080062, P91=0.0017436533887164587, P92=0.0013586747838424398, P93=0.00045694136041360607, P94=0.0005566948441702961, P95=0.0005449584111252103, P96=0.0008558438908617161, P97=0.0004066142294753439, P98=0.00026739174644177223, P99=0.00014516537928523638, P100=0.00013836470998941823, P101=7.035177057551485e-05, P102=7.403597207539432e-05, P103=9.450287747062077e-05, P104=0.00014703171918206722, P105=0.0001576464146115844, P106=0.0001338624571535805, P107=9.44223640400543e-05, P108=0.00015489979959801815, P109=0.00021964471065354153, P110=0.00021836375809499735, P111=0.00015742964622208976, P112=0.00014168418623356594, P113=0.00012693546856074107, P114=8.660359475101543e-05, P115=0.00020858813945236275, P116=5.734024606150889e-05, P117=0.00013611052594452794, P118=0.00013358266891103372, P119=5.0762180221593474e-05, P120=0.00014472130423640137, P121=6.970902818957728e-05, P122=9.743232942581243e-05, P123=7.522409128674225e-05, P124=4.193576731729496e-05, P125=4.631073734524521e-05, P126=2.422398241740836e-05, P127=1.0130663256106695e-05, P128=1.1990984943921404e-05, P129=1.369483443417892e-05, P130=1.0526507728493852e-05, P131=2.489051411359784e-05, P132=3.796550533084938e-05, P133=4.299320474760491e-05, P134=3.757439772580951e-05, P135=3.314508768597597e-05, P136=1.886136624130496e-05, P137=1.3354397980303787e-05, P138=1.0787370788319787e-05, P139=4.955124238910504e-06, P140=4.692337794461283e-06, P141=7.01970108105811e-06, P142=7.44518394047939e-06, P143=7.531432069741955e-06, P144=1.3673523105907979e-05, P145=1.7026543743110968e-05, P146=9.942029981348392e-06, P147=5.173127215874267e-06, P148=2.254936898579138e-06, P149=2.5601313759143613e-06, P150=4.005621560130834e-06, P151=5.074598264571829e-06, P152=1.3632129325255018e-05, P153=1.64106381579301e-05, P154=3.198983531546088e-05, P155=4.248051003984929e-05, P156=4.6276842559810396e-05, P157=3.4584346064708593e-05, P158=3.729377908257633e-05, P159=4.3653317618745754e-05, P160=2.8657614037168964e-05, P161=2.384253573903768e-05, P162=1.2207792267689896e-05, P163=8.206311209416243e-06, P164=4.390742757872581e-06, P165=2.177876638883848e-06, P166=3.2436505970870053e-06, P167=4.103261155410783e-06, P168=5.666636286356631e-06, P169=2.938032848086543e-06, P170=1.1423184777711732e-06, P171=1.74269691141516e-06, P172=1.7485807555233552e-06, P173=9.792340052851465e-07, P174=1.193504221843241e-06, P175=2.4341338909649133e-06, P176=3.504393056065507e-06, P177=4.877211208996389e-06, P178=3.6203206287036635e-06, P179=2.957520263828633e-07),Row( P0=-523.1486521724657, P1=37.67980255066833, P2=-16.431754627492428, P3=-13.856873841991021, P4=-18.453834499338633, P5=-17.029546721437576, P6=-17.2178927828237, P7=-4.330780707553587, P8=-6.624924016276054, P9=-11.874509000461321, P10=-7.758263780699881, P11=-5.160090905821524, P12=-7.193994751369195, P13=-6.5349324091096594, P14=-5.747497968841754, P15=-1.4039086250965878, P16=1.1460083928185765, P17=1.8396109907569964, P18=0.03788307820340006, P19=4.922252848325107, P20=-1.9477709218104127, P21=0.8340235895562585, P22=0.08693387503730825, P23=1.0060229206143523, P24=-0.15877248803135155, P25=2.415783657798421, P26=-0.6083527051849436, P27=-0.5417182001656287, P28=-0.8443484154417001, P29=0.09177707016776511, P30=-1.5013098051601232, P31=0.8957329438320198, P32=-2.3609083366414545, P33=0.2060497329784329, P34=-1.0268476725857734, P35=-0.17773209916254282, P36=-1.993341311043061, P37=-0.7555520318586156, P38=-1.1095318641872998, P39=0.34655127567705657, P40=0.6891826378351273, P41=0.6111686264883717, P42=0.6375961899349737, P43=0.6662800624172439, P44=0.6787664722327438, P45=0.6871364329752842, P46=0.694343302158015, P47=0.6734461866757344, P48=0.6988711907582483, P49=0.7097521142053206, P50=0.6914812095053455, P51=0.6900349351543298, P52=3.308650053159711e-07, P53=4.659942823310013e-06, P54=1.6798547363003055e-05, P55=4.033001682848801e-05, P56=9.904857134859085e-05, P57=0.00013437057544011863, P58=0.0003008391909720043, P59=0.0010721542455067224, P60=0.0075130750837973485, P61=0.04111834053872795, P62=0.09872394514324591, P63=0.12634670471429302, P64=0.1081974821801558, P65=0.21141788975558762, P66=0.29971010602512166, P67=0.2627033275461475, P68=0.1379820711775353, P69=0.12113100568033826, P70=0.16527085084287083, P71=0.10758502513037108, P72=0.12422410172183178, P73=0.11159064011285866, P74=0.04411719754632123, P75=0.022748685703158085, P76=0.05137985083434105, P77=0.18631886014486126, P78=0.30974575748632005, P79=0.40507394598872337, P80=0.5889874588282238, P81=0.4610853218025873, P82=0.3620217229460628, P83=0.5762616448472313, P84=1.2255750338286795, P85=0.6306256024028242, P86=0.057649004675694546, P87=0.03843983322041668, P88=0.05669671713053869, P89=0.05560258164773376, P90=0.06009395185243258, P91=0.060753462985966615, P92=0.058808982350625885, P93=0.03019027150235676, P94=0.06079257832346495, P95=0.07287354657530033, P96=0.09893821109732108, P97=0.03416101033826861, P98=0.015984202170738825, P99=0.008618412599759649, P100=0.005608946569365762, P101=0.00359185975727964, P102=0.0024089838305256742, P103=0.003747128674001407, P104=0.005625494491553522, P105=0.009756156826129174, P106=0.006963054848323157, P107=0.0026567724118415474, P108=0.005051013568931695, P109=0.010740536632059035, P110=0.021374850644851808, P111=0.02980685104320667, P112=0.02089685223796337, P113=0.007520297771138258, P114=0.00237427865546086, P115=0.0033986487113453626, P116=0.007594250558159537, P117=0.013001463498084558, P118=0.011940110116776053, P119=0.005102812130463964, P120=0.004725994719729058, P121=0.003507797207622073, P122=0.005900543991320209, P123=0.004396937913644794, P124=0.0036485556621886745, P125=0.006413977581442138, P126=0.0035701016271138916, P127=0.004269883402450736, P128=0.0020916686154638232, P129=0.0030102028861862195, P130=0.0024867125992424103, P131=0.0020811506582351534, P132=0.002450934380260274, P133=0.0015809053894348698, P134=0.0016084461958625275, P135=0.0016152964243975956, P136=0.0007715892394657259, P137=0.0003231022135166036, P138=0.00011128806292831226, P139=0.00013263850936538735, P140=0.00024906159000678206, P141=0.0003634294837107719, P142=0.0004983976733262209, P143=0.00038256507419322255, P144=0.0005663621562458319, P145=0.0006896258221473425, P146=0.0009907498042312304, P147=0.0008381143106027458, P148=0.0015827258886271715, P149=0.0011998617300614418, P150=0.0009639615687300738, P151=0.0010648353699904355, P152=0.0007668092881481801, P153=0.000423035521688387, P154=0.00045222201825501757, P155=0.00041745648347255395, P156=0.000550151088722175, P157=0.0006046951796400718, P158=0.0005651694646278406, P159=0.0007056856351908922, P160=0.0006811396321754768, P161=0.0007318740476482382, P162=0.0005980164592884984, P163=0.0004945454053084483, P164=0.0006079216985773141, P165=0.0010484050327728676, P166=0.0011504243311105974, P167=0.001578388262259087, P168=0.0019894140156892165, P169=0.002493171453923631, P170=0.0018992513701801734, P171=0.0013974878140869957, P172=0.0013815309414534648, P173=0.0013938358638820225, P174=0.001042834428499787, P175=0.0012248579258750463, P176=0.001154445893800363, P177=0.0004765305803967819, P178=0.000220028110104772, P179=3.168940509313214e-05),Row( P0=-705.0623793845824, P1=64.1573870295856, P2=-12.702334114216733, P3=6.202539057742756, P4=-11.685908164915588, P5=-11.599557338981173, P6=-8.638307738842778, P7=0.08024116989061039, P8=0.0783027946077424, P9=-4.647190046276224, P10=-10.58928770108745, P11=-5.692385509147894, P12=-0.5561092800231924, P13=-6.094125748276304, P14=-5.298701747852368, P15=-5.590257936341743, P16=-1.301540490586562, P17=-6.648413368589368, P18=-7.103280400118822, P19=-4.402019285595242, P20=-8.293306430344801, P21=-5.9080095009599995, P22=-3.566472080826681, P23=-4.3244028995906705, P24=-9.445048017360616, P25=-3.2688365044332044, P26=-6.511444930708471, P27=-5.826419608309181, P28=-3.108506882694314, P29=-4.4647306263059425, P30=-2.5971946010546194, P31=0.5231414869132361, P32=-0.9973157450191948, P33=4.884156790932487, P34=1.9639640122923927, P35=3.070438797869487, P36=-0.2554009376606219, P37=1.6439594231036019, P38=0.04312729725456943, P39=0.6357630705118871, P40=0.6813978646540579, P41=0.6496330585145755, P42=0.7163563700865608, P43=0.7510303640651795, P44=0.7309309182687143, P45=0.7076716902697232, P46=0.7242052413928391, P47=0.7141356754034868, P48=0.730072364878634, P49=0.7135058480819636, P50=0.7048757303992385, P51=0.7302994087069765, P52=2.1335483713271022e-07, P53=7.039428599775363e-07, P54=6.046645543808764e-06, P55=3.104565325039504e-05, P56=0.0008778262017434501, P57=0.007287849021515299, P58=0.017536734283477115, P59=0.030048440956334785, P60=0.009125502176233794, P61=0.0010306658713347156, P62=0.0003543958400574727, P63=0.0003222019144464844, P64=0.0006201146983991312, P65=0.00458583053409652, P66=0.01501939383456207, P67=0.020550086617976134, P68=0.008598193017992818, P69=0.002480707410781624, P70=0.0013510754222360497, P71=0.0010597334494623595, P72=0.0020543598247073345, P73=0.0026468581544071778, P74=0.0038161366933382066, P75=0.0044845975222904755, P76=0.002736135583490056, P77=0.0033151837966364393, P78=0.0031254405479518824, P79=0.0024751868975645915, P80=0.0019278211320592094, P81=0.0018644118525040641, P82=0.003736162991363161, P83=0.005198037344318884, P84=0.003902378698226395, P85=0.0020516791274275956, P86=0.0017835024177279188, P87=0.003183508846033828, P88=0.004260975135853756, P89=0.013826584706144155, P90=0.01961545666466156, P91=0.0023378529121666418, P92=0.001601179728013156, P93=0.0005915624105742983, P94=0.0009202171084530177, P95=0.0031301744039165225, P96=0.001181097517345281, P97=0.0006643984585712195, P98=0.00033042052836601885, P99=0.00035557467100826896, P100=0.0008183182417395596, P101=0.00015343188530462167, P102=1.3344232942553987e-05, P103=3.857146952722649e-05, P104=0.0001942240989522228, P105=9.744560177218803e-05, P106=6.481861892497006e-05, P107=0.0001349432058035901, P108=0.0001928647308581559, P109=5.8043475921574366e-05, P110=0.0018060889780892625, P111=0.004114269464722389, P112=0.0002716272020912096, P113=0.0003435422434908631, P114=0.00046547798292430895, P115=0.0001345706083682947, P116=0.0003019684800713563, P117=0.0002953006237487162, P118=0.0001415009419556006, P119=0.00015450314922491198, P120=0.0002483457642597967, P121=0.00010802220558136231, P122=0.000176299060961316, P123=0.00013477996527070212, P124=3.203262444190066e-05, P125=2.5990580492558397e-05, P126=2.2469942508527944e-05, P127=1.2921928321113513e-05, P128=1.1280870853322077e-05, P129=2.5047034527509387e-05, P130=1.9543176333867673e-05, P131=3.1334428518588874e-05, P132=5.8117437096712136e-05, P133=9.02133715871038e-05, P134=6.102113761250721e-05, P135=7.911077237593744e-05, P136=2.477607237221527e-05, P137=1.025008661745923e-05, P138=1.1431659458509516e-05, P139=5.374992883807927e-06, P140=7.406258746111947e-06, P141=9.25870393231597e-06, P142=1.7437299454611826e-05, P143=2.2679740177487973e-05, P144=2.3405766062251605e-05, P145=2.9031926133180457e-05, P146=3.770881392329753e-05, P147=3.5514151235700435e-05, P148=9.372486246719561e-06, P149=7.747476682376093e-06, P150=1.8931095348696778e-05, P151=5.786716690685593e-05, P152=0.00010450826128391888, P153=0.00014594067973212185, P154=0.0001696870459114741, P155=0.00014092566023827404, P156=0.00011339157111168084, P157=0.00013086106973378393, P158=9.397617063780766e-05, P159=0.00011619261691791725, P160=0.00017340110091259135, P161=9.295187393059025e-05, P162=5.421426452725028e-05, P163=3.21333852312859e-05, P164=1.696555391851352e-05, P165=1.556674176641909e-05, P166=1.3756725161533627e-05, P167=2.5318954420635935e-05, P168=1.8127274623482193e-05, P169=1.0834382512043734e-05, P170=8.414573094048504e-06, P171=8.825707244226351e-06, P172=8.093844398012123e-06, P173=4.843630287328498e-06, P174=4.234771866489253e-06, P175=6.121748786437828e-06, P176=7.002947892222315e-06, P177=7.292487646718668e-06, P178=4.242111377038204e-06, P179=5.35593033303795e-07),Row( P0=-459.59602894056627, P1=37.23159515643785, P2=-29.347344421712382, P3=-6.8339952445947905, P4=-16.011660657467633, P5=-19.932197613146506, P6=-20.300995023582267, P7=-0.9236679694096537, P8=-2.384227322905881, P9=-6.299743813469853, P10=-12.307119225556233, P11=-4.591916520577486, P12=5.827125048927326, P13=-0.8207447466496641, P14=7.504055035493716, P15=13.178242724091245, P16=12.110854042966846, P17=6.451629373270149, P18=-1.7168507215114026, P19=7.438313560579928, P20=-1.3805280655966747, P21=-0.09441860912553177, P22=4.7613513309915, P23=-0.1847641861393838, P24=-2.4853698056941917, P25=0.6098923605538058, P26=-1.3962610852857726, P27=0.6323016261052963, P28=1.3237405497033878, P29=1.780713574941999, P30=-0.3588379619337667, P31=2.006157167215361, P32=-2.1003807754860415, P33=-0.000932270719981911, P34=-2.4341128885038574, P35=-2.066775909595986, P36=-1.3211568179121176, P37=-2.1163611290401763, P38=-1.6261214762691285, P39=-0.6608307757696814, P40=0.6222986396487161, P41=0.5749474867766763, P42=0.6145734364315745, P43=0.594764514868178, P44=0.5560406120960888, P45=0.5975573176287209, P46=0.6687756236416056, P47=0.629998162614198, P48=0.6340566952623652, P49=0.6476681917191435, P50=0.6067783050311931, P51=0.6017134703281712, P52=0.0009930866804395188, P53=0.0010348108345640836, P54=0.0005236966573000192, P55=0.0003171910763131428, P56=0.000212310594161245, P57=0.0001566944183022802, P58=7.33292341417756e-05, P59=0.0001350567915647871, P60=0.0010684797610819021, P61=0.007169285579758585, P62=0.05234536144602696, P63=0.09402470471478652, P64=0.08947614915690727, P65=0.2649233599878661, P66=0.6275347571544763, P67=1.0823874800446367, P68=1.4855215428066224, P69=1.9780748355145155, P70=3.0788866278139837, P71=1.8272735225837697, P72=0.42911574705887195, P73=0.7801979518773384, P74=0.4627675475467473, P75=0.08998236633244004, P76=0.034047981792724974, P77=0.026453325483773216, P78=0.04057253774547177, P79=0.06719238338568582, P80=0.09422337293266753, P81=0.3717257576151909, P82=0.5090102012318442, P83=0.7235974649735695, P84=1.0209413848138342, P85=0.627208662854274, P86=0.8968896546994871, P87=1.6478103976553007, P88=1.7975403737981037, P89=1.2053568466891402, P90=0.9789636495055306, P91=0.7744565515285369, P92=0.1775705119697858, P93=0.08216498064573528, P94=0.10957306911580204, P95=0.09529225931319267, P96=0.3070445825581103, P97=0.18860883880109347, P98=0.10211912578305439, P99=0.14415162501369735, P100=0.1555003577476615, P101=0.23340319190349648, P102=0.4289745907195444, P103=0.584344958010736, P104=0.13449662146403144, P105=0.026163906609995808, P106=0.031163152816844522, P107=0.027218854721145293, P108=0.05544579918389462, P109=0.32330110453327415, P110=0.9853524884414747, P111=1.063199246126001, P112=1.1716216219810558, P113=0.2474693209547676, P114=0.0887115376785358, P115=0.06347697428652227, P116=0.08901015393547877, P117=0.09009271404212035, P118=0.06563558978461846, P119=0.030527242913332225, P120=0.03651154921421972, P121=0.05112326932506095, P122=0.04931955711986015, P123=0.07887501351763929, P124=0.012531230146114614, P125=0.006381819774617339, P126=0.012455237370408287, P127=0.01792749659379808, P128=0.010532712593185958, P129=0.009253591672071923, P130=0.011296479672265648, P131=0.02256673570191591, P132=0.023626709967690743, P133=0.04940896873071377, P134=0.027337303705399073, P135=0.05367079828910773, P136=0.030076200520129253, P137=0.004881220955011026, P138=0.0029702872518397286, P139=0.005324755563643117, P140=0.003723975299260978, P141=0.00706775949000297, P142=0.008648338767604232, P143=0.014765553324808164, P144=0.010746037123671321, P145=0.021009771423426884, P146=0.01071046497269042, P147=0.004548421429242667, P148=0.002628923713435539, P149=0.001028323290171435, P150=0.0019427962151985908, P151=0.0020591148213833337, P152=0.002930185537909845, P153=0.0026620255945062226, P154=0.003338119940871316, P155=0.0031069775146967886, P156=0.0017892954708371724, P157=0.0036297777509201675, P158=0.007118253931049312, P159=0.006346537635582775, P160=0.005758013206817517, P161=0.002226263707549529, P162=0.0009824406866792005, P163=0.0007502698827566035, P164=0.0006530458780875511, P165=0.0006035387130613537, P166=0.0006972322307939253, P167=0.0005913123880582408, P168=0.00041968027152524437, P169=0.0004904283336509664, P170=0.000587459097107034, P171=0.0007768001308916109, P172=0.0007572632335475838, P173=0.0005387077802301386, P174=0.00039764038155296023, P175=0.0004646950854095338, P176=0.0006491629364155921, P177=0.0006770215053445744, P178=0.00045939214914705124, P179=5.2356021113216215e-05),Row( P0=-592.1041594198775, P1=49.6976371325545, P2=-19.51772204961898, P3=-8.144081366388445, P4=-16.163926614808148, P5=-15.308454632473245, P6=-18.459278871180015, P7=0.8577097543995654, P8=-10.577978394867968, P9=-10.031746997045113, P10=-8.78697720864549, P11=-13.22460290532016, P12=-4.31765595833537, P13=-14.256928192669879, P14=-6.451387803483173, P15=-5.444733110684974, P16=-3.414750734575353, P17=-0.14668446359288984, P18=-1.30047674864376, P19=1.7548549877568218, P20=-3.781493954192968, P21=-1.6423585963698062, P22=-1.0855013417316266, P23=-4.517122568186312, P24=-4.666346181405635, P25=-1.1150615661414554, P26=-3.1202123362750087, P27=-1.369727343961664, P28=-2.5014967812755087, P29=-1.4825819317156668, P30=-4.767420354576639, P31=-2.8032205923483673, P32=-3.7517428575025016, P33=0.619781333349284, P34=-0.15054616464840417, P35=-1.1057327874746967, P36=-1.1695565084363337, P37=-0.08645886005686845, P38=0.8043421425630666, P39=1.611173182236418, P40=0.48844443619445344, P41=0.4325337851817328, P42=0.3929180495838696, P43=0.43565296122023217, P44=0.46224384849231187, P45=0.4765994256890419, P46=0.48698536174082907, P47=0.47064598327987495, P48=0.5292848078651182, P49=0.5718692231424356, P50=0.5580624244900015, P51=0.5373855940081603, P52=2.8169830730101633e-07, P53=2.129688631343516e-07, P54=2.1670228453383595e-06, P55=8.389438720897988e-05, P56=0.0005621921262117885, P57=0.001604539888941106, P58=0.0024457173748199377, P59=0.0033359715192163536, P60=0.00749282413087569, P61=0.022035117528816487, P62=0.03405651573161325, P63=0.03949548134902823, P64=0.04049335909797594, P65=0.06628319406135583, P66=0.1125845860085099, P67=0.07206684305616357, P68=0.019515647149297186, P69=0.010130918907100004, P70=0.01447495511619561, P71=0.012774540751284159, P72=0.010572754187268238, P73=0.014908314599574906, P74=0.012841090537085192, P75=0.01163748107953135, P76=0.022560509368884187, P77=0.0956029065140304, P78=0.12728288446313316, P79=0.10295080634531599, P80=0.10980449379864635, P81=0.09930210515282764, P82=0.07603697275999609, P83=0.0821344998269021, P84=0.07596206877710932, P85=0.052355110850847535, P86=0.031566759943766065, P87=0.023364828962144143, P88=0.07625277911619956, P89=0.12077032779754912, P90=0.06613981955813342, P91=0.02967676801013357, P92=0.031722551901098746, P93=0.016654715670549637, P94=0.014591115643615139, P95=0.011606087887008546, P96=0.010015534956878252, P97=0.1720477406024568, P98=0.2226225231249126, P99=0.035074696243130817, P100=0.007362549904259903, P101=0.004719880456309809, P102=0.0019395934653714697, P103=0.0015274897190638115, P104=0.0033923715461058905, P105=0.004516472468742302, P106=0.004086323036429251, P107=0.003836405527866923, P108=0.001520994231021515, P109=0.0027383093429108915, P110=0.015989702045945856, P111=0.017422363095960555, P112=0.008566626546142986, P113=0.00339470906019384, P114=0.0054023132612150224, P115=0.014961502015979702, P116=0.0049069207782355476, P117=0.0015048204850224526, P118=0.0011662886048293731, P119=0.0017613661017673297, P120=0.005132253232606612, P121=0.003137666639405733, P122=0.001172425933545354, P123=0.00232246148378986, P124=0.0019361951719531717, P125=0.000611047405364139, P126=0.0015862161276139403, P127=0.0008128863548155777, P128=0.000271197458840921, P129=0.0008022071833574655, P130=0.0005655105087809271, P131=0.0009774907490389697, P132=0.0013197547629227647, P133=0.0007088760893848998, P134=0.0007394836363117876, P135=0.0010373590000301695, P136=0.0006429109529995816, P137=0.00013743370175839527, P138=0.00012006653105551426, P139=6.769763751445077e-05, P140=6.528499403439369e-05, P141=6.516964303397706e-05, P142=6.890087711251318e-05, P143=8.862468108029691e-05, P144=9.99308095912976e-05, P145=0.00016319490524237598, P146=8.145640070356412e-05, P147=6.452732962311288e-05, P148=7.586725594611162e-05, P149=8.94326035422968e-05, P150=9.514703555002323e-05, P151=7.961903100919122e-05, P152=7.588787193893052e-05, P153=9.349513481841376e-05, P154=0.0001228484590524248, P155=0.00015895317336474355, P156=0.0002472703856859152, P157=0.00033124899815895135, P158=0.00041114527948544246, P159=0.0006950736785414446, P160=0.0006984442979957888, P161=0.000745939866408328, P162=0.0011659175793187998, P163=0.0005215927028842331, P164=0.00033192328680514054, P165=0.0003548362217970942, P166=0.0005392156682253118, P167=0.00037178674416073654, P168=0.00024259710764357276, P169=8.449273597192959e-05, P170=5.2638514243058224e-05, P171=5.169529019519775e-05, P172=6.415660344727913e-05, P173=7.91704344782192e-05, P174=6.532643785364301e-05, P175=4.951055584297912e-05, P176=4.0219915861828104e-05, P177=7.659778850069e-05, P178=5.500746715132865e-05, P179=6.161801694377704e-06),Row( P0=-428.16103675395703, P1=31.419654902515987, P2=-35.11069603462782, P3=-12.42919297489239, P4=-20.08184429515279, P5=-11.769350019298606, P6=-18.998676318477035, P7=-4.083250563457809, P8=-7.501568489799861, P9=-9.361563727144766, P10=-10.362636989484162, P11=-11.988199923861249, P12=6.777314910583486, P13=0.1985287327364263, P14=14.072202678098597, P15=18.685210691934802, P16=10.795739056767133, P17=14.12954370031502, P18=-0.1711570677261522, P19=-0.17536234389696448, P20=-3.1653397282918263, P21=5.722801276679214, P22=5.798976435432305, P23=1.11215944210896, P24=-2.7306435195907044, P25=1.388458657026765, P26=-2.9064351694816186, P27=8.522054545460302, P28=3.847574644276695, P29=5.438381571795849, P30=2.034309567998106, P31=1.3265930273960447, P32=1.181142692722591, P33=4.285903427201741, P34=-0.2839382454537622, P35=-1.7610186536798815, P36=-1.302802047562377, P37=2.7847823045184477, P38=2.2028486849138567, P39=3.4125276513012857, P40=0.5488981786589004, P41=0.6110428599562396, P42=0.6171326570349699, P43=0.6411053458355312, P44=0.6330613384089162, P45=0.6217655449382813, P46=0.5361139617884985, P47=0.5126132342216755, P48=0.5492482164670517, P49=0.5584184975703436, P50=0.542589136181627, P51=0.5369542851476384, P52=0.010404594713663964, P53=0.001977557040716306, P54=0.0002835149708064338, P55=6.858572451267524e-05, P56=8.065265952095392e-05, P57=0.00012890695889017527, P58=0.0002298365375465013, P59=0.0023242061593699522, P60=0.012624316164670829, P61=0.017606027404469645, P62=0.0185033481023664, P63=0.03135372404096364, P64=0.06726261250420096, P65=0.1915848136580261, P66=0.39716998775346324, P67=0.5279065335810805, P68=0.8782961663376879, P69=0.9696003773807295, P70=0.8327033293076188, P71=0.5725887559285971, P72=0.16454895650722157, P73=0.05895049551693933, P74=0.019850418038351327, P75=0.01035821840347248, P76=0.010898425004428924, P77=0.02211970631298871, P78=0.028279812821926865, P79=0.04189655707687214, P80=0.096534371927205, P81=0.26279309907733056, P82=0.4904501620506474, P83=0.7577562531682277, P84=1.9937981400744171, P85=7.7863983583617005, P86=5.513497331100412, P87=1.7148867667489998, P88=1.5241327065282984, P89=0.7001524958890567, P90=0.46826294175176203, P91=0.45336741710077755, P92=0.4054737275171237, P93=0.2197711651411759, P94=0.1961074997926968, P95=0.230059532822406, P96=0.4213294738696072, P97=2.9020934000183476, P98=4.471295605044285, P99=3.605738248481223, P100=1.345660211659151, P101=0.7315931896591447, P102=0.12870236502703317, P103=0.2422524839655795, P104=0.1501586033582114, P105=0.14949825193653132, P106=0.6583394909450802, P107=0.49133807692569, P108=0.16014714944019517, P109=0.49745580909107623, P110=0.37113551009012363, P111=0.1593959954677257, P112=0.15971088975205333, P113=1.2562436341159546, P114=1.6725645979895303, P115=0.7971066254722038, P116=0.08607388479454908, P117=0.09597228905366435, P118=0.061756956043639566, P119=0.04990662378064754, P120=0.2445837549471132, P121=0.21082994977763794, P122=0.05239320362680905, P123=0.028739698342142912, P124=0.044709581232063, P125=0.09161050895765221, P126=0.032544579435678705, P127=0.06628031571940865, P128=0.04659626066443197, P129=0.017096485146156932, P130=0.028215332520238307, P131=0.009527975942011607, P132=0.017465431781061084, P133=0.013221641911789655, P134=0.014604620396638856, P135=0.03146113493998834, P136=0.039625188612741356, P137=0.015398579751291455, P138=0.008009217791100184, P139=0.004992052033050204, P140=0.003748171078367446, P141=0.005545060775036454, P142=0.003355314259608735, P143=0.0027246044480754347, P144=0.006371920900079302, P145=0.014062495619366313, P146=0.02063795648537565, P147=0.025567389285848764, P148=0.01195284559226711, P149=0.0028133299610762913, P150=0.002353632259031805, P151=0.0009649082342586579, P152=0.002203180739995683, P153=0.0037246688482835207, P154=0.0073982493172729, P155=0.016129697662543345, P156=0.013288876281148406, P157=0.019945328746715073, P158=0.015888019051797042, P159=0.0092777843195972, P160=0.004021996504842914, P161=0.003359862741879442, P162=0.0022156803938952756, P163=0.0017413432001205133, P164=0.0012273968262873705, P165=0.0014860729551649217, P166=0.0012397728689976365, P167=0.0011416146532657984, P168=0.0015058445067054522, P169=0.0013722813764547371, P170=0.0009230768059177577, P171=0.0010838785464452854, P172=0.0011370465021742907, P173=0.0012388703280664661, P174=0.0014356345773426512, P175=0.0016445952758750763, P176=0.0018377601320907937, P177=0.0016566609095081996, P178=0.0006600421410674853, P179=6.785782604388543e-05),Row( P0=-642.3510712809243, P1=59.465916930412014, P2=-12.097731523086534, P3=-0.5037581594384125, P4=-13.265005662864239, P5=-12.769111570965533, P6=-5.907727598357232, P7=0.833276683348369, P8=-10.413598878121828, P9=-8.81904414653015, P10=-6.031242360356543, P11=-8.983956835008318, P12=-2.2304355906474043, P13=-10.741347081328476, P14=-5.813353810251915, P15=-4.282583832759326, P16=-6.903243191479703, P17=-0.34224276726262176, P18=-8.092678663796919, P19=-4.356801134104825, P20=-8.51941840649362, P21=-6.198879849758991, P22=-3.97001959647958, P23=-6.852652394725702, P24=-2.9544719065568197, P25=-3.610087061497739, P26=-3.025055026935906, P27=0.33365270857830154, P28=1.2920619018695823, P29=2.4570700188899064, P30=1.1225859575195436, P31=4.017655097540486, P32=0.030297532528897227, P33=3.001469047342432, P34=-0.41744469712282367, P35=0.15707402852993047, P36=-0.7357901623888121, P37=1.1822498116335434, P38=1.8251485386803457, P39=2.9557572657600732, P40=0.6033588558948786, P41=0.6055195893239904, P42=0.5589227799867232, P43=0.5741120671143954, P44=0.5446207021342327, P45=0.5350909593001781, P46=0.5467254226126587, P47=0.5552569267552055, P48=0.6169853712163146, P49=0.5981291631815203, P50=0.5806370433545399, P51=0.5640501018062668, P52=2.15873226670131e-06, P53=4.285058737069073e-06, P54=8.771139485283598e-06, P55=2.2510246109706975e-05, P56=0.0003915450539961218, P57=0.00562751393164162, P58=0.007216678191810473, P59=0.012374018774706404, P60=0.018684775471774284, P61=0.012420485942516678, P62=0.003123539581389003, P63=0.0012252848892667428, P64=0.0021911678212143456, P65=0.003786077847404591, P66=0.00562325689617872, P67=0.009947903230017442, P68=0.022297464983048576, P69=0.02027750582630298, P70=0.04178309084434978, P71=0.028777596652270775, P72=0.001456089729634866, P73=0.0010969949943261598, P74=0.0010039234587610238, P75=0.0014016173287927868, P76=0.004144972428461907, P77=0.011458871933927325, P78=0.012885370622078743, P79=0.020305693385884338, P80=0.051392621078969546, P81=0.026260735874949166, P82=0.004193174117618161, P83=0.0025589739912218207, P84=0.0056915982890442, P85=0.006797726788349859, P86=0.003433433497004401, P87=0.001962842339461602, P88=0.0029987323775704827, P89=0.003096759323033242, P90=0.0029822832838905505, P91=0.004371610353326754, P92=0.007874924852803913, P93=0.005239201393066821, P94=0.004153014761041897, P95=0.005375687254294437, P96=0.0060665085248642345, P97=0.006685060217817386, P98=0.008411778992147312, P99=0.006605229402103372, P100=0.004651457139565984, P101=0.0031995931871343385, P102=0.0032739766282423362, P103=0.001763846044646199, P104=0.0015344272576484263, P105=0.0013311655223729323, P106=0.002095568119795937, P107=0.0030602194057038506, P108=0.0016975136504200205, P109=0.000395213133300461, P110=0.0013511682576314013, P111=0.0031621845350758813, P112=0.0010941196265118617, P113=0.0005866248456548224, P114=0.0006041926985179112, P115=0.00040608170816027996, P116=0.00022834253121918743, P117=0.0007687604228704776, P118=0.0010936392602025984, P119=0.00033763773930902807, P120=0.0002538684923380649, P121=0.00046944857011512126, P122=0.00020265734599250986, P123=0.0008248804048910821, P124=0.0006239555338099348, P125=0.00027309994537331355, P126=0.00017576471771325124, P127=0.0004536413830786581, P128=0.00038204637626299714, P129=0.0001791858478156607, P130=0.0001474020325124194, P131=0.00015358978203510294, P132=0.0001502817487541462, P133=0.00027546011957522916, P134=0.00038486318399544225, P135=0.0002449002130474254, P136=0.00027030654244626775, P137=0.00022287460017768608, P138=6.824098748554433e-05, P139=5.4969950617263924e-05, P140=5.46187784012894e-05, P141=4.459138179267834e-05, P142=6.325898635362956e-05, P143=5.582657081861476e-05, P144=8.211362203335395e-05, P145=5.384127229762464e-05, P146=3.247065366815929e-05, P147=1.3275450992912179e-05, P148=1.232524473831856e-05, P149=1.6944182888863395e-05, P150=1.8923769638934986e-05, P151=1.6324774003245365e-05, P152=2.2535068783540692e-05, P153=4.769807981793599e-05, P154=3.0176367203951525e-05, P155=3.3308186728724477e-05, P156=6.6361476830502e-05, P157=6.258795066617786e-05, P158=9.312912519441166e-05, P159=0.00012487638229508247, P160=0.00013094499138992406, P161=8.350661355909403e-05, P162=4.775233899057586e-05, P163=2.296024633265778e-05, P164=1.7039563451470974e-05, P165=2.739812794080172e-05, P166=2.0033159556274466e-05, P167=2.1902720021293745e-05, P168=2.1566950777602312e-05, P169=1.46741668276284e-05, P170=1.0991942779787463e-05, P171=8.375883297665354e-06, P172=2.0252222214431828e-05, P173=2.5712960696119537e-05, P174=1.5820006624633145e-05, P175=7.701746398190689e-06, P176=7.137065466340468e-06, P177=1.0533520797031591e-05, P178=5.933727564793573e-06, P179=8.464000763748297e-07),Row( P0=-563.6809236207806, P1=58.110813888802674, P2=-23.394131189413, P3=1.0344581264973078, P4=-16.224373266262806, P5=-13.09438692840826, P6=-7.7957099533889345, P7=-6.482687407997843, P8=-10.618629061300151, P9=-11.93634772783044, P10=-12.058040112289275, P11=-4.9582299481715175, P12=-6.078230657762973, P13=-11.977820630005969, P14=-6.546843791034769, P15=-6.033359249776337, P16=-2.9067599272075397, P17=-2.7749418443081875, P18=-3.675053563098507, P19=-2.047861065128359, P20=-2.744380503892837, P21=-1.2884079413645269, P22=-1.3134823659944979, P23=-1.8239211432901585, P24=-2.1310666285958457, P25=-0.8385932721521464, P26=1.4890652145213727, P27=-0.15694447677166676, P28=1.7076876524981717, P29=3.962132508448912, P30=-0.13966849662510347, P31=-2.0219231882313626, P32=-1.785289158850591, P33=-1.005109516309663, P34=-4.07512895938256, P35=0.05354181037657548, P36=-5.03074119421954, P37=-1.0975932135825495, P38=-3.3471407851085377, P39=-0.847295239105653, P40=0.6726125808519151, P41=0.6301861659872107, P42=0.6245949790911273, P43=0.6854715659708104, P44=0.6989497525599967, P45=0.6717169815256198, P46=0.6830542211574651, P47=0.6928273067027096, P48=0.7433595510584049, P49=0.7463058991759929, P50=0.7410961181038184, P51=0.7078185303424579, P52=7.558253215271545e-06, P53=6.894653913339554e-06, P54=2.0135411328137664e-05, P55=3.601809508700339e-05, P56=0.00011003202967450945, P57=0.00040858453950743545, P58=0.0008154439572284785, P59=0.003936732053827965, P60=0.01991778646459891, P61=0.03123064396839994, P62=0.02761980948480041, P63=0.03998652259073651, P64=0.035802389043282666, P65=0.012727473390981165, P66=0.0032641169187857706, P67=0.00409417830410648, P68=0.010351944016938992, P69=0.022732013047144548, P70=0.04119875853578804, P71=0.027059492440055746, P72=0.011549123764056505, P73=0.005421431459409087, P74=0.005059173553245692, P75=0.011361629833877344, P76=0.05073196789490745, P77=0.06396730107565649, P78=0.02735309504680815, P79=0.02212557883775101, P80=0.018337966373868392, P81=0.01968852282599588, P82=0.02586732998864028, P83=0.011901645643726345, P84=0.00771453036803701, P85=0.010081337221597856, P86=0.008627556516035402, P87=0.011932175523980012, P88=0.03432182347967972, P89=0.02393313420171471, P90=0.01407804910850593, P91=0.010698944784892042, P92=0.010432255485346207, P93=0.008983329531035786, P94=0.05603141451659042, P95=0.09293592644822186, P96=0.06643039740638865, P97=0.06564728784095669, P98=0.05413363460376732, P99=0.015104410537433355, P100=0.015596210956924388, P101=0.016910379913109303, P102=0.013050696191655237, P103=0.009933367419415003, P104=0.012975751984291933, P105=0.013282460663980675, P106=0.01418768373726401, P107=0.031046203266953905, P108=0.02916919208620758, P109=0.011167175614981975, P110=0.012717429678994285, P111=0.029624758942395087, P112=0.006968647995077744, P113=0.005060345582085718, P114=0.0015418952916228992, P115=0.001973266297348189, P116=0.008752930072351887, P117=0.0030659436493006413, P118=0.0010851059173084096, P119=0.002315096301527466, P120=0.002038978399293528, P121=0.0010271329602172146, P122=0.00313610763685741, P123=0.0010950158139649989, P124=0.001021904623978003, P125=0.0010572792666299115, P126=0.00098005049857346, P127=0.0008895182060455099, P128=0.0006097969741228475, P129=0.0007300168508676375, P130=0.0009069693232910214, P131=0.0013867014386537784, P132=0.0016433028629797841, P133=0.002124416733186119, P134=0.002586276376279096, P135=0.0011044872434440647, P136=0.0013205342373661972, P137=0.0009758442480872771, P138=0.0007398507069959939, P139=0.0007460643302448321, P140=0.0006506396160128559, P141=0.0007587865686006853, P142=0.000756393693763321, P143=0.0008842822005533644, P144=0.001166500643524339, P145=0.0006769014214187082, P146=0.00034546303825666374, P147=0.0002186367084211518, P148=0.00017219208424619156, P149=0.00012901915598398337, P150=0.00014308298633788704, P151=0.00019508410640967596, P152=0.00024683424497247, P153=0.00025171598500016505, P154=0.0002878065547321782, P155=0.0002183894880584346, P156=0.00011882370240085749, P157=0.00016031781152061776, P158=0.00015241327439201053, P159=0.0001983279426257296, P160=0.00022060929648032648, P161=0.00043831796206831615, P162=0.0004028043637340226, P163=0.00023887831271614826, P164=0.0001689006221476252, P165=0.0002753223038673529, P166=0.00023916509376863736, P167=0.0001402611722062552, P168=0.000168556086133204, P169=0.00023743827011857743, P170=0.0002473896034494361, P171=0.00013889399300045846, P172=8.228391685250287e-05, P173=6.298588416505746e-05, P174=0.00010950692871325893, P175=0.00013354332307092233, P176=9.8742861108796e-05, P177=0.00012119420591510003, P178=0.00014155757049134232, P179=1.2797032254000604e-05),Row( P0=-374.5798023538906, P1=39.29614855464414, P2=-43.428351096165976, P3=-11.741276986557365, P4=-22.23150708695449, P5=-17.913061254341994, P6=-18.28581584707977, P7=6.665449787820684, P8=0.009491011448484956, P9=1.0205936500469361, P10=-1.1979154766207185, P11=-6.200841966582789, P12=17.180824228233025, P13=5.229081202932242, P14=14.804986404860145, P15=26.477759636191735, P16=12.69106145803108, P17=8.501676992073813, P18=1.2436801223550007, P19=2.9581457782823697, P20=0.8079875902954311, P21=-0.21688771220249142, P22=4.310856541001891, P23=0.39939244950514596, P24=-5.538318939807655, P25=5.155275280392202, P26=2.8748570482112012, P27=5.438922368129561, P28=8.390983399180573, P29=6.942252049441108, P30=0.06290192704530871, P31=6.248282637321836, P32=1.883290666385172, P33=2.2950277344508967, P34=-0.18225999975799, P35=4.047889887196013, P36=1.8705747500439918, P37=3.283418274485065, P38=0.5420830378243146, P39=4.293219815061233, P40=0.5584962355151605, P41=0.6053491436716384, P42=0.7442496827225289, P43=0.7427793867984005, P44=0.6451315530515622, P45=0.6023030856654029, P46=0.5986352288319646, P47=0.5695327039971313, P48=0.5726239453805345, P49=0.6081825021536189, P50=0.5706134793638471, P51=0.547365323779586, P52=0.3838108364473909, P53=0.05152923623538725, P54=0.005651466748910253, P55=0.002259590462619922, P56=0.0013392801925518859, P57=0.0007225234693714851, P58=0.0003882649933701411, P59=0.0005201412351774649, P60=0.0006849848392736631, P61=0.0007786262043268133, P62=0.0017249059102718452, P63=0.0037492520811417385, P64=0.006610418637454404, P65=0.04707930589994291, P66=0.2218462296997584, P67=0.5159024933957206, P68=1.054972979941508, P69=2.363236639702974, P70=3.5162690364459004, P71=2.339346153696502, P72=0.8371457808243609, P73=0.44612004567038205, P74=0.17137918344018924, P75=0.07354296442084854, P76=0.07589725323852385, P77=0.15417141583901992, P78=0.3377696031451005, P79=0.6155464754145366, P80=1.0843132753381926, P81=2.3827375738977103, P82=3.0085425290380776, P83=2.3928706716714707, P84=2.1319216928931404, P85=1.5896686501128983, P86=1.38829045545245, P87=1.7810770954158957, P88=4.874035586515294, P89=3.9913526590667887, P90=2.7361208487921163, P91=1.3353644302486773, P92=0.2619458572623602, P93=0.08102811134735943, P94=0.11297630804274074, P95=0.14487923382547113, P96=0.18865465441047363, P97=0.768392770117267, P98=1.869580261541117, P99=1.4404290691041282, P100=1.0657353359522772, P101=1.116233011029702, P102=0.8053579553731207, P103=1.5759739811068125, P104=0.2889966580413104, P105=0.1594830547230855, P106=0.35457808216533965, P107=0.6110083073392517, P108=0.6476019940041338, P109=0.6252057888563061, P110=0.8683569081348546, P111=0.573356327445699, P112=0.12193017605086899, P113=0.14819396550745953, P114=0.16620454480289787, P115=0.17203050313638196, P116=0.0912663545781392, P117=0.14821510136273264, P118=0.17939362271447995, P119=0.18925334776974614, P120=0.1649895082634061, P121=0.1959195345009327, P122=0.29073348915533226, P123=0.15497679621893154, P124=0.07024841833301164, P125=0.0337603447049656, P126=0.10895388115471462, P127=0.05292242776455976, P128=0.029961494183407253, P129=0.05299596265198081, P130=0.05514004004425157, P131=0.02965321868644765, P132=0.0448949187744813, P133=0.07574231575938234, P134=0.09610915651300185, P135=0.034235340815259356, P136=0.015461981523067462, P137=0.01399045649247315, P138=0.009893035940808502, P139=0.008240756129219488, P140=0.01919440867354723, P141=0.01714018087114523, P142=0.02267376574540522, P143=0.0231732154943132, P144=0.012242068553409934, P145=0.0060558062452408905, P146=0.0022590914793859246, P147=0.002351889557004765, P148=0.005073514982211864, P149=0.010882077845004461, P150=0.013302832327407943, P151=0.015013788314826375, P152=0.005301018475493457, P153=0.006991425823750199, P154=0.0074545867737148665, P155=0.008431776372405338, P156=0.011697761474090797, P157=0.014555252441183782, P158=0.013777941634588061, P159=0.009911234071689048, P160=0.008567281355318757, P161=0.005697454619988355, P162=0.0040731575381526145, P163=0.004297958916055069, P164=0.004157475477091748, P165=0.0031122421184748344, P166=0.0020143645221799545, P167=0.002342961091859549, P168=0.0024212314575736567, P169=0.002621095955508325, P170=0.002822591370517762, P171=0.0033880675742573897, P172=0.003988486564420682, P173=0.004952740318558248, P174=0.004639662494771123, P175=0.0033172571941762047, P176=0.003938932459496796, P177=0.0032822218758764613, P178=0.0014771884652955564, P179=0.00018772192956135336),Row( P0=-530.9011421614387, P1=49.05847913428761, P2=-25.58505423059131, P3=-1.203116893808903, P4=-17.95072297239226, P5=-18.08689039704153, P6=-9.026095390546406, P7=-2.2035851033374927, P8=-10.149903171188798, P9=-9.948520266044774, P10=-10.130859692044254, P11=-5.7390184685165995, P12=-3.6473155857860493, P13=-11.989825899283174, P14=-6.495475856185751, P15=-5.282371990252157, P16=-0.6976563220674641, P17=-4.3628975331097894, P18=-3.576428601178055, P19=-7.102326197780634, P20=-6.987697028899768, P21=-5.720901742521755, P22=-5.6699403048145784, P23=-3.2957186693113703, P24=-1.9120578300999211, P25=3.2142097000717658, P26=4.904790091054907, P27=3.658526933351653, P28=3.6984125174231552, P29=2.6233324154410256, P30=-1.5019974708158228, P31=-1.0026768240522839, P32=0.15597014115059313, P33=2.390259701427123, P34=-1.137665921112906, P35=1.089197412595854, P36=-3.124960052893072, P37=-1.7964741448872799, P38=-1.3100968123459715, P39=1.5664783626620864, P40=0.675946967978121, P41=0.6038279512399922, P42=0.6064900469871408, P43=0.6811234528347421, P44=0.7107379613669067, P45=0.6812481706838242, P46=0.633552177754863, P47=0.6489502660799997, P48=0.6798235748924492, P49=0.6897173230514901, P50=0.714987961582873, P51=0.7203110531600737, P52=2.5244684757137177e-05, P53=1.769396237293244e-05, P54=3.746462889906075e-05, P55=4.1773463199337964e-05, P56=0.00011725424714881943, P57=0.0007736486033286876, P58=0.0017031647048514258, P59=0.005108987666502519, P60=0.03283948017343713, P61=0.09853015763045338, P62=0.06935275088872646, P63=0.0077172286757814, P64=0.0007041338946864371, P65=0.001333817948819268, P66=0.003266763236475993, P67=0.006294029938219734, P68=0.00845285433212522, P69=0.021982277510049632, P70=0.05745747774578957, P71=0.08438465159858696, P72=0.1130158318488047, P73=0.08678897327747535, P74=0.019263745371438407, P75=0.0021723605326834064, P76=0.0033492813401383697, P77=0.005042441404696885, P78=0.010996923652838156, P79=0.023729053758311098, P80=0.04464068259143777, P81=0.09438446029898503, P82=0.17877534295740335, P83=0.28625029388433243, P84=0.3542334242059382, P85=0.08110663883264818, P86=0.007980074223182934, P87=0.015698026988546537, P88=0.05233511136041397, P89=0.04973237758211182, P90=0.036605670732529766, P91=0.02964649345892556, P92=0.03451136940026648, P93=0.01618924797687892, P94=0.009095835531195558, P95=0.011510322811465155, P96=0.031059663407584214, P97=0.03979675320772133, P98=0.02950130746452298, P99=0.03197937046591468, P100=0.009603958359543679, P101=0.009529626536907322, P102=0.018293131839903255, P103=0.00988247108528072, P104=0.002021847580722438, P105=0.0013309828034549875, P106=0.002632202517601266, P107=0.006048690346395426, P108=0.0033540036505299692, P109=0.0010856533090864956, P110=0.001379177330642666, P111=0.007665654689750932, P112=0.005018575512021655, P113=0.0023113829394536232, P114=0.0023955052736638225, P115=0.00886621019027092, P116=0.002611707769629566, P117=0.0015607991811490813, P118=0.008126564255734206, P119=0.00407106482818915, P120=0.0011638148864397918, P121=0.005894000121497877, P122=0.00319817170203894, P123=0.0031329535910633328, P124=0.004133624913840505, P125=0.006258362633645663, P126=0.0029439236527717652, P127=0.0027219892646217847, P128=0.0017161552621873962, P129=0.0019818789537081787, P130=0.0019357955173598066, P131=0.001300098444035786, P132=0.0017208865399448586, P133=0.0017089038720351172, P134=0.002968614522276987, P135=0.002128751009841181, P136=0.0036759049868127338, P137=0.0018604530810958644, P138=0.001783716379158263, P139=0.0020998079648236723, P140=0.0011660035641042248, P141=0.0010133584852145537, P142=0.0004080145528934395, P143=0.0002586333781692914, P144=0.00030621487573104917, P145=0.0002361103468860074, P146=0.00021530178175715805, P147=0.0001675529620966379, P148=0.0002284211565693356, P149=0.0002295445917389309, P150=0.00016436084520389082, P151=0.00011986261839438919, P152=0.00014081417836776794, P153=0.00013026130953128204, P154=0.000148516477895402, P155=0.00015983189052539988, P156=0.00016353581382163912, P157=0.00017617346146033032, P158=0.00019550692386475824, P159=0.0003179399098793611, P160=0.000507563480168771, P161=0.0007138063182282044, P162=0.0008227755041875372, P163=0.0005450189547619242, P164=0.0004906657877632039, P165=0.00042798268126748013, P166=0.0002769583549030277, P167=0.000155600807386457, P168=0.00044352065935374346, P169=0.00031981302378732525, P170=0.0002771422039728118, P171=0.00019232762845286516, P172=7.899806929014612e-05, P173=6.380386654019009e-05, P174=6.117653750224898e-05, P175=7.215253156918687e-05, P176=6.395640352459931e-05, P177=8.843368895408993e-05, P178=5.4474282558844744e-05, P179=7.142177413429031e-06),Row( P0=-608.634360975696, P1=50.51257081502321, P2=-8.706508308830866, P3=-0.6292938059850476, P4=-17.97780469019471, P5=-15.02534691118689, P6=-8.08491311363907, P7=-1.0178988973322465, P8=-11.595496157701314, P9=-9.679434003636233, P10=-7.373136078156899, P11=-5.364101574144867, P12=-6.80081833029494, P13=-11.862961105194326, P14=-3.1133612444264536, P15=-2.5289450409283476, P16=-1.7475376356465613, P17=-2.022911069554247, P18=-2.4981402467016505, P19=-4.65697174804624, P20=-5.061817007425676, P21=-5.592140859581156, P22=-3.6251185933639216, P23=-2.368800854237114, P24=-3.6359243085600244, P25=2.806032353861673, P26=0.7043670146670319, P27=2.0511571559431343, P28=2.2427575244992033, P29=2.068426365392449, P30=1.6324832041131474, P31=0.4735518802345912, P32=2.674205939716097, P33=1.207241912530349, P34=0.06493420872088411, P35=2.610339409801353, P36=-0.7105413667259659, P37=0.969640435567936, P38=-0.48420467774280074, P39=1.0315895382093754, P40=0.770959930585452, P41=0.6980024011542574, P42=0.6789203173141127, P43=0.6956645339578538, P44=0.6696458500217427, P45=0.6463098737862154, P46=0.6533119305741557, P47=0.6364850390663288, P48=0.661962331195665, P49=0.7233064563542028, P50=0.7844278665286868, P51=0.8090747081310666, P52=6.322539396378712e-05, P53=2.7484228022749165e-06, P54=3.3629764179249834e-06, P55=1.0751916458471111e-05, P56=7.87380814647581e-05, P57=0.0005618624065710254, P58=0.0017482356926291382, P59=0.005490105572730372, P60=0.017903998301479394, P61=0.0315735055492811, P62=0.03178299838795169, P63=0.012333774779342228, P64=0.0009454610011590232, P65=0.0014671125526848911, P66=0.003832519200126322, P67=0.006496968153406903, P68=0.01092505109735839, P69=0.024938885297886976, P70=0.04218928052674067, P71=0.03255184222146992, P72=0.01753873591163836, P73=0.02140439655328416, P74=0.013582949077817276, P75=0.0014116879542130705, P76=0.0009569457861603045, P77=0.0027556232284669632, P78=0.005669624738150249, P79=0.015537024663969029, P80=0.03140599736849384, P81=0.05754174502406434, P82=0.0648904370504969, P83=0.02669765875229502, P84=0.01926253249590816, P85=0.011969431986203738, P86=0.004485261696805193, P87=0.004063970548726885, P88=0.008615991951444734, P89=0.004453616834376848, P90=0.002133534625865107, P91=0.0028449941075753753, P92=0.005910954806435635, P93=0.010057862512829144, P94=0.0052533797621325045, P95=0.0042503851761305205, P96=0.003761523529523999, P97=0.003291375373925595, P98=0.002949688534620114, P99=0.0034852016346188053, P100=0.0037017152738473944, P101=0.0032924976654729755, P102=0.002582881245708073, P103=0.002067818456305431, P104=0.0014678465416299187, P105=0.0022896636389517374, P106=0.003855043764212185, P107=0.001119022772263677, P108=0.0005150444643941205, P109=0.00029884254840611973, P110=0.002504371388319826, P111=0.007333001743932381, P112=0.001191255555614012, P113=0.0006505904273631983, P114=0.000775456559558038, P115=0.0017372650448740066, P116=0.0005385012877084584, P117=0.0006959268885513486, P118=0.0015598506578933997, P119=0.0005538244824743856, P120=0.00041204746057273405, P121=0.0007816277779144553, P122=0.0006500521633467434, P123=0.0007667618976918193, P124=0.0012663943110267394, P125=0.0008499341374243971, P126=0.0007653381404416542, P127=0.0006933327762610017, P128=0.0002962823321370939, P129=0.0005570877248459174, P130=0.000481958751472913, P131=0.00039716876476211517, P132=0.00048430162730308824, P133=0.00044563479922263906, P134=0.000443983986677917, P135=0.0003214481951486413, P136=0.0006454848026560927, P137=0.0004545154498669299, P138=0.000387328282721843, P139=0.0004419691482362562, P140=0.00039409902644273604, P141=0.0004140555293129698, P142=0.00024633335035209423, P143=0.00014187774662503198, P144=0.00015976396835117707, P145=0.00013349502338659344, P146=0.00010533480368496845, P147=8.013161449821738e-05, P148=9.341130793145055e-05, P149=0.00010252096397912486, P150=0.00011644384453700638, P151=0.00012725390548565642, P152=0.00019516786578804927, P153=0.00028082659319934264, P154=0.0002989710557378739, P155=0.0002792229028645747, P156=0.00022010053843294115, P157=0.0002491969744922925, P158=0.0002626074972573652, P159=0.00041523579981859554, P160=0.00046236324448806047, P161=0.0006319891314288086, P162=0.0006599717650662889, P163=0.0008622764446801048, P164=0.0006911969307038754, P165=0.000647099142594122, P166=0.00033382120863318613, P167=0.00012520527085657868, P168=0.0001855874490301007, P169=0.00021662853101682173, P170=0.00020939416364462358, P171=0.0002527612010785212, P172=0.00013676703466819628, P173=0.00012803524995273454, P174=6.93570322025982e-05, P175=6.702534103776681e-05, P176=9.048368432481655e-05, P177=0.00011710230751927033, P178=7.356999804015211e-05, P179=1.0123943325266336e-05),Row( P0=-598.4066374563998, P1=62.126769821484324, P2=-15.968840425062277, P3=-0.5499963598576251, P4=-15.913800786109912, P5=-17.649605727132652, P6=-8.035957224408456, P7=-4.994710441197943, P8=-5.8386719651232, P9=-9.095565774062925, P10=-9.761344429057253, P11=-5.299286013733222, P12=-5.441663122846738, P13=-10.542947408351932, P14=-5.836157971460278, P15=-4.023637390131471, P16=-1.1933560194416066, P17=-2.4725908083977606, P18=-3.3111346066482628, P19=-3.7649695389645244, P20=-5.887047595370294, P21=-4.814482163549545, P22=-7.009217021888782, P23=-4.826068945792528, P24=-4.873253158830135, P25=-1.3785053078075529, P26=-0.32467951126756134, P27=0.7999904497213229, P28=1.8910908640536925, P29=2.116644742305862, P30=1.168280352545203, P31=1.634507267717185, P32=-1.0166065062621863, P33=2.088489322019178, P34=-2.0371294901336245, P35=2.4803446931620674, P36=-0.5023473442948804, P37=2.1340614284895962, P38=-0.42749246088588166, P39=1.1199447294696794, P40=0.6961790405063649, P41=0.6574291383235367, P42=0.6922450781319612, P43=0.6759812594648698, P44=0.6689387490660713, P45=0.6816056675762858, P46=0.6927524889239489, P47=0.6756647650405356, P48=0.7389397547824962, P49=0.8076533612276602, P50=0.7748647043330956, P51=0.7350463142128881, P52=1.654679385469352e-05, P53=6.123833159173118e-06, P54=9.098726566054047e-06, P55=2.3382015882110796e-05, P56=0.00018413960286118566, P57=0.001040164641049933, P58=0.0020325815058143595, P59=0.006043485131644666, P60=0.015028018501396952, P61=0.01869387167985614, P62=0.00924765503486859, P63=0.0016198126541188533, P64=0.0008337166645469295, P65=0.002048615511133989, P66=0.006098335021499352, P67=0.012024080681778064, P68=0.01602153459772505, P69=0.0279975284614549, P70=0.04625761584525243, P71=0.02526928317517156, P72=0.015711168805901777, P73=0.00855266635807855, P74=0.002261698427495462, P75=0.0022777668094353313, P76=0.003324069032239223, P77=0.01557971724518851, P78=0.05494739465941709, P79=0.11963524792860392, P80=0.06342122309883033, P81=0.022181050912568483, P82=0.019092441604246526, P83=0.010346870627849801, P84=0.005894412148093979, P85=0.00893580793705326, P86=0.009755061637535435, P87=0.0065141917167330825, P88=0.008070516774655414, P89=0.004798372554106198, P90=0.002910072402805687, P91=0.0033367457102250458, P92=0.005670596623439599, P93=0.0042129986690051125, P94=0.009139069950878457, P95=0.024802947594076162, P96=0.009869213086650565, P97=0.0029359246506415367, P98=0.002991466230136268, P99=0.004568128402292586, P100=0.013320598021417279, P101=0.012284760231644978, P102=0.003084866761093741, P103=0.0008533086424464226, P104=0.0007894009596605457, P105=0.0032439039700929285, P106=0.003246614255751509, P107=0.001789500551890226, P108=0.0007636797430091696, P109=0.002414677601548634, P110=0.001947627682470705, P111=0.002521592710668297, P112=0.0011934328886448694, P113=0.0007246160004773651, P114=0.00036512191436958964, P115=0.00039570155461014566, P116=0.0004275534643993358, P117=0.0006773054681368275, P118=0.00038686876545519777, P119=0.00041713176260510466, P120=0.0002540733942174376, P121=0.0002984303848238458, P122=0.0002917229223835907, P123=0.0001744123570466555, P124=0.00021882490048228174, P125=0.000313702027870243, P126=0.0003655844998174571, P127=0.0005633446983629251, P128=0.0006388040519222891, P129=0.000528540681505974, P130=0.00032504015401620085, P131=0.0005768369024222419, P132=0.0008778921844495163, P133=0.0010413439729559297, P134=0.0010949121306826474, P135=0.0004534443294417287, P136=0.0005044580907436019, P137=0.0004175417215695418, P138=0.0005441869399930993, P139=0.0004791390706998909, P140=0.000358637116972717, P141=0.00042861293256824255, P142=0.0003789577523074378, P143=0.0002801376998487346, P144=0.0003005194389939419, P145=0.00021675538537421733, P146=0.00020452426381391191, P147=0.00017570518456137474, P148=0.0001337613307100186, P149=0.00010023140997514591, P150=0.00012348455628479613, P151=0.00010159740334117454, P152=0.00012955238497500008, P153=0.00015372370899616934, P154=0.000162123921032054, P155=0.0001458149200343216, P156=0.00010142250207147995, P157=6.817436957710408e-05, P158=5.630678350643959e-05, P159=0.00014531740392681285, P160=0.00013611280578945946, P161=0.0001905781137252617, P162=0.00016107832068688993, P163=0.0001745893731454222, P164=0.00012079468621070925, P165=6.487215320385758e-05, P166=0.00012623612227940088, P167=6.569919117114902e-05, P168=0.0001387552006899875, P169=0.00012203498929797483, P170=0.00011323267299928594, P171=6.99582672771922e-05, P172=3.9413588272315894e-05, P173=3.7525905162851286e-05, P174=3.7681479555318105e-05, P175=5.029624843343902e-05, P176=6.46522944057024e-05, P177=5.598283385027567e-05, P178=2.5869048543113416e-05, P179=3.811675954293821e-06),Row( P0=-537.4217175302945, P1=65.89969606611893, P2=-18.277763181136027, P3=-2.177005539420574, P4=-18.04033106957841, P5=-7.83176939865011, P6=-5.290698091736589, P7=-3.3460467658971544, P8=-12.297419159549781, P9=-8.41852698992474, P10=-9.300812477288707, P11=-9.790262461231157, P12=-4.749146226865078, P13=-11.261654433185305, P14=-7.7725000144030485, P15=-6.862177097761468, P16=-4.650362336112825, P17=-3.42554605945296, P18=-4.548259924996546, P19=-7.611961389585801, P20=-5.361420161534785, P21=-5.336165680188845, P22=-2.018311122223613, P23=-2.180259084542464, P24=-4.2845953633079885, P25=2.179218862975933, P26=1.7779152675381928, P27=0.3770175503958279, P28=-0.74261068592185, P29=2.538893269391719, P30=1.3790796389580868, P31=-0.21011926605707543, P32=-0.39446385198816625, P33=1.6078616718484955, P34=-0.7309395505311055, P35=0.5926247644138369, P36=-3.5711565070241873, P37=1.6566322119160193, P38=-2.4425974064328226, P39=0.9914136054814563, P40=0.6530595639094351, P41=0.5934892529933776, P42=0.6653875269376957, P43=0.7170533190062235, P44=0.7218349390679424, P45=0.7002603353192869, P46=0.6607387574169082, P47=0.6519487628784605, P48=0.6907292043097636, P49=0.6971404766800346, P50=0.6515363039512165, P51=0.6655683378031725, P52=2.968453181425906e-05, P53=1.5861934843903483e-05, P54=2.0657759356816056e-05, P55=2.7400866556044452e-05, P56=0.0002454907500367802, P57=0.0022701002573026225, P58=0.002609407086223777, P59=0.004765657155835755, P60=0.01707341175114729, P61=0.033035627486441024, P62=0.07680165046289043, P63=0.10021878053567074, P64=0.02301454059750833, P65=0.002042672251952291, P66=0.0030212013532065858, P67=0.005283275016438161, P68=0.008081854052155, P69=0.01405930219834839, P70=0.023937095447439397, P71=0.017506751548345797, P72=0.024248037173421992, P73=0.0420392525401252, P74=0.03859603417901858, P75=0.03931882692298537, P76=0.01797672703106613, P77=0.006024156266875843, P78=0.00890904228199788, P79=0.016513086799828356, P80=0.015245318158267766, P81=0.012973897560798758, P82=0.019038104772591182, P83=0.04293746517896019, P84=0.06271896574362514, P85=0.027571884258466184, P86=0.00621078923373159, P87=0.008056149422723887, P88=0.00891099905815007, P89=0.0049226372620242605, P90=0.004650008251487368, P91=0.005230383046513293, P92=0.008282573634113383, P93=0.006760476193824261, P94=0.00467204307886025, P95=0.00883670850143126, P96=0.021829813860422615, P97=0.012924513884720455, P98=0.009447634301383703, P99=0.010322580184024004, P100=0.005341614216367048, P101=0.005109086660125454, P102=0.004610381141759949, P103=0.003594033919650883, P104=0.003066053786620428, P105=0.0028200421394460086, P106=0.009079572860777508, P107=0.006577303601549616, P108=0.002172578631950008, P109=0.0012649485229355105, P110=0.007892979948274422, P111=0.011276365950465689, P112=0.0026350766216278036, P113=0.0018300598579918104, P114=0.001077396688364289, P115=0.0004758860675242719, P116=0.00032424105200893244, P117=0.0005720728189781095, P118=0.00043123275079142944, P119=0.0003228153134771832, P120=0.0005748033249303961, P121=0.0008077530235775949, P122=0.0007454927566724056, P123=0.0004821911626578525, P124=0.0006456406312528773, P125=0.0004431764950759345, P126=0.0007138694851393519, P127=0.0008489636907572426, P128=0.00047238309792782394, P129=0.0006283156271286864, P130=0.0005894547755913498, P131=0.0003799956631794579, P132=0.0004431021530596826, P133=0.00037786936416618215, P134=0.0004981546540008383, P135=0.000483852137341598, P136=0.0007724448698028073, P137=0.00042441333884480763, P138=0.0004074139587419077, P139=0.00042932200228039906, P140=0.0009634157832976734, P141=0.0007466083098462488, P142=0.0004657659162026031, P143=0.00041589837676345447, P144=0.00023505850459523402, P145=0.00021093618581287274, P146=0.00021529558409906732, P147=0.00010868996888985566, P148=0.0001294436102282213, P149=0.0001292345393112141, P150=0.0002304720823403155, P151=0.00015175700147893227, P152=0.0003855566978772662, P153=0.0004399794700447463, P154=0.0004273078273606183, P155=0.0004499787406033389, P156=0.00015922197914603161, P157=0.00015718389807182486, P158=0.00013841044315374335, P159=0.00013512164393969297, P160=0.0002675992858324374, P161=0.00036734500688752103, P162=0.00032889540445769894, P163=0.0005600700271872404, P164=0.000448508953499687, P165=0.00020861378506349356, P166=0.00016477464033348355, P167=0.00011932656121153498, P168=8.001654603303899e-05, P169=0.00014176834726033953, P170=0.00012194012489827445, P171=7.45423947089501e-05, P172=5.280543517850511e-05, P173=6.487964712488918e-05, P174=7.344730926770836e-05, P175=8.224604586893076e-05, P176=0.00014460393663957437, P177=0.0001423435715774907, P178=8.510469543239234e-05, P179=1.2149539003179602e-05),Row( P0=-379.9288182094594, P1=35.393909343853935, P2=-22.638430038382644, P3=-16.145815444655334, P4=-20.787683281481062, P5=-15.526469446028205, P6=-22.20161246433245, P7=-5.514342410574945, P8=-3.5428493929044698, P9=-4.717990100799706, P10=-3.04746489143234, P11=-6.676888750503944, P12=8.541285400699937, P13=6.627862921371522, P14=7.586881534295048, P15=14.081883140334247, P16=3.3445809703388845, P17=5.167597733874848, P18=5.409292318858753, P19=7.2906667696248135, P20=3.16141157346248, P21=5.078820531711722, P22=2.6085870862387295, P23=3.711403638709869, P24=6.2131832966835425, P25=7.70445338638096, P26=1.5560175322295875, P27=3.4970646704867314, P28=2.289614944791104, P29=3.257338456764877, P30=1.8016318381363945, P31=2.5526320820414825, P32=-0.7370530009918745, P33=1.5424851782505244, P34=2.6871747481781414, P35=2.585495660392236, P36=-0.7296368812088224, P37=0.7875180310497292, P38=0.8879110519022023, P39=3.961745554690675, P40=0.6356978760220664, P41=0.5420396054713111, P42=0.6049530564455252, P43=0.6481178658620104, P44=0.6440805019635318, P45=0.6398205512691248, P46=0.5921125644674424, P47=0.5325300109371714, P48=0.5048032449504557, P49=0.5267328015256822, P50=0.576205022306496, P51=0.6262240121046981, P52=0.06512368644089561, P53=0.022808024815277328, P54=0.0028981993269730864, P55=0.00024673177457200417, P56=0.00017620228867130155, P57=0.00016938331536799946, P58=0.00021454710048147342, P59=0.0006285525535594442, P60=0.0035273513534714775, P61=0.01830693506984079, P62=0.05259920511178156, P63=0.07025053616899087, P64=0.05152974243954049, P65=0.10118681786857663, P66=0.19957881595253402, P67=0.3063173083777789, P68=0.45024212691857685, P69=0.8626192690495595, P70=1.478967345821835, P71=1.6838416327125898, P72=1.2701697591323065, P73=1.4845506346090287, P74=1.5062697130274525, P75=0.8860342198358524, P76=0.3741605057455961, P77=0.4163776509558722, P78=0.600121913582702, P79=0.6972213003882626, P80=0.5097266636552601, P81=0.42163333497410516, P82=0.6598088840760803, P83=0.9091867485223655, P84=1.6811595102330714, P85=1.4801315547217802, P86=1.2265509565011274, P87=1.8805986658497522, P88=4.574752865243975, P89=3.7586226836702474, P90=3.7584816207756715, P91=4.807002782051168, P92=5.960812803377839, P93=3.16114109990838, P94=3.9973471739167157, P95=1.7321406314012615, P96=0.2380539977770659, P97=0.25598468912876426, P98=0.39196679480175134, P99=0.48696883418833176, P100=0.36105944747618135, P101=0.25007016146138983, P102=0.25341773253091365, P103=0.6621128995725377, P104=1.4991594105031445, P105=5.326856426584972, P106=3.8361057973088415, P107=0.7672198147710553, P108=0.1927753264189633, P109=0.1957745414714714, P110=0.3198672638597144, P111=0.3868706713938853, P112=0.110014327708674, P113=0.29672132273751745, P114=0.1324991428359326, P115=0.10753509684551675, P116=0.15064268039355244, P117=0.25794890445603225, P118=0.3321554769113253, P119=0.15499235612821072, P120=0.08203488978169296, P121=0.0654391020933237, P122=0.14894915888825713, P123=0.37262605460055315, P124=0.1407599006270355, P125=0.037677057864538274, P126=0.04827375991040519, P127=0.03890038732611093, P128=0.035177310758464475, P129=0.02000691287916313, P130=0.015856861492628015, P131=0.016833187198953724, P132=0.03413361523277263, P133=0.03687525901543475, P134=0.04194636908790763, P135=0.07281516692173812, P136=0.05534575016090713, P137=0.023854305287483037, P138=0.014189085519752901, P139=0.00971720445252487, P140=0.007468117043039701, P141=0.007941336257227835, P142=0.0066236628836564134, P143=0.008076605036468862, P144=0.006331467964641112, P145=0.006707549485380767, P146=0.008228283740018034, P147=0.009188899160775369, P148=0.014480191422143152, P149=0.012349065040148141, P150=0.011945762267108117, P151=0.04316828865579787, P152=0.020166613202074572, P153=0.01768610183609247, P154=0.019701907078089616, P155=0.017738020887675156, P156=0.016653761889757996, P157=0.01906152162086019, P158=0.01668975537534946, P159=0.014215082076858447, P160=0.010464060568164733, P161=0.009306777974898503, P162=0.006304535199653897, P163=0.006234963873820123, P164=0.00433603774508589, P165=0.004591444570254194, P166=0.004725411246847691, P167=0.005751343677427037, P168=0.005202090224776206, P169=0.005927372482826446, P170=0.0040437856945319565, P171=0.004194460620481656, P172=0.0043121130335276306, P173=0.004712366950707531, P174=0.00396440451625437, P175=0.003451339683681211, P176=0.004878910775409165, P177=0.0052198227676385055, P178=0.003364568976301171, P179=0.00042862809917768014),Row( P0=-416.7617678444897, P1=46.5306811813751, P2=-17.093736130745192, P3=0.8833561576344287, P4=-15.126421447803226, P5=-16.277852500320343, P6=-19.046849573487876, P7=0.012353315951532476, P8=1.5632627211065053, P9=-3.5674655506303963, P10=-0.528981495496971, P11=-1.065516381448082, P12=11.304425417813215, P13=5.1950600455852385, P14=11.920478160664581, P15=22.73991497401859, P16=15.381272349251837, P17=14.779936894061489, P18=5.154958566538532, P19=8.460578541297787, P20=3.4656213004509966, P21=3.248194023261801, P22=6.256887497995921, P23=4.693196420096293, P24=-1.6991366549096205, P25=5.663296318711442, P26=2.4552086853065154, P27=2.900900326358614, P28=5.221956465436383, P29=3.902509201263971, P30=-1.163851233157646, P31=1.1698221034805785, P32=0.05247757020377977, P33=-0.11415660371060579, P34=0.15548593780723236, P35=2.1348682475212413, P36=-0.21649100682689124, P37=3.6821235925792544, P38=0.5744880574006183, P39=4.143246896704715, P40=0.6015742141807834, P41=0.5798322705895963, P42=0.6622355096040469, P43=0.6651910224680702, P44=0.624871277472474, P45=0.6032358027151592, P46=0.5999101829525643, P47=0.5999158777669525, P48=0.6341144117272762, P49=0.6948686419100741, P50=0.6454418251942284, P51=0.634867376680972, P52=0.16852896275545468, P53=0.028640165103066943, P54=0.0075240512876453575, P55=0.0018308194851052915, P56=0.0007275242854070611, P57=0.0007565749652567034, P58=0.0004341959879590172, P59=0.00018070455793228561, P60=0.00025792525863993504, P61=0.0005015159556079064, P62=0.004756811520618421, P63=0.014794958463638395, P64=0.02545067802918124, P65=0.12267667072531498, P66=0.37273181334795036, P67=0.6904696794152839, P68=0.8338937312609966, P69=1.5228456029493675, P70=3.0443472286703486, P71=2.2796918152368835, P72=0.7672860912333023, P73=0.3503357437782782, P74=0.08724944286343157, P75=0.05012609173296824, P76=0.043598570870620586, P77=0.06666092236612939, P78=0.16173705872943703, P79=0.3677918238203046, P80=0.5196801093490898, P81=0.6686332619185706, P82=0.9241361286175584, P83=0.7712112349603665, P84=1.0076544180699383, P85=1.2482379845712854, P86=1.5942626942123952, P87=1.9931970194817072, P88=3.034958588509091, P89=1.4095293783483234, P90=0.5354511851901581, P91=0.7050187154714268, P92=0.6822828980718457, P93=0.08898767237040109, P94=0.07704246569869964, P95=0.3910275056573497, P96=0.5895802663342323, P97=0.3111751655578878, P98=0.2685476225150805, P99=0.27974983684066435, P100=0.511984543180869, P101=0.4679626632432214, P102=0.20740317150880702, P103=0.30154559605305054, P104=0.4730226045718381, P105=0.19967041793990806, P106=0.12114059870520585, P107=0.13792672633677955, P108=0.10036945147658512, P109=0.061132041175811266, P110=0.10184818417842932, P111=0.31372615078170174, P112=0.24411920008846982, P113=0.1374942048974434, P114=0.1478699833443946, P115=0.2517365980001702, P116=0.18296718162098122, P117=0.42049940542329944, P118=0.16306807240680812, P119=0.10763014713036352, P120=0.07407087823136473, P121=0.1376269356157299, P122=0.16435772764113102, P123=0.14332354257104563, P124=0.14814150945508267, P125=0.12242824642788203, P126=0.024810862559882775, P127=0.022721408997178012, P128=0.028641523333114486, P129=0.05081935174453967, P130=0.019330609643133254, P131=0.017008962324518782, P132=0.025746487409430696, P133=0.04078736384746929, P134=0.04141084118066385, P135=0.04079094283698578, P136=0.02460086404927405, P137=0.012649072190640322, P138=0.010147587978127398, P139=0.007050617803568056, P140=0.011703146551305655, P141=0.00862116283633353, P142=0.010117271992036, P143=0.009542837492232628, P144=0.014269233167795853, P145=0.005651730820917418, P146=0.0028037530414528637, P147=0.0024959655678554325, P148=0.0029624830641064846, P149=0.005927640446975981, P150=0.010225817516505444, P151=0.01463082452697609, P152=0.014451319165331935, P153=0.009559719196470022, P154=0.007588107723319265, P155=0.007982691866777204, P156=0.010752301767650567, P157=0.009846390951309396, P158=0.005513289489618272, P159=0.006316145458122102, P160=0.005479900497547872, P161=0.004589183819802748, P162=0.0031010974122060043, P163=0.002715722209886399, P164=0.002836834532140466, P165=0.002119098349394546, P166=0.001825328048727168, P167=0.001566977551239933, P168=0.0022592311286456835, P169=0.0025925045935960896, P170=0.004077436004888093, P171=0.0040122648982872135, P172=0.0035949663253887023, P173=0.004399361571245369, P174=0.004875610893315882, P175=0.002718303156644143, P176=0.002878433511413629, P177=0.003950218962593361, P178=0.002020023211680715, P179=0.00021992653174943776),Row( P0=-522.9979003376693, P1=62.72401998573766, P2=-20.397763672341558, P3=-10.774472979026923, P4=-20.464651875642158, P5=-12.87266829429205, P6=-12.025540996204017, P7=-7.350361092247479, P8=-17.095932584691916, P9=-6.092140001745995, P10=-10.13538157523479, P11=-13.495916841310196, P12=-6.6885822363442236, P13=-8.93358432660034, P14=-9.36404426364973, P15=-4.788485037956424, P16=-5.745987996561839, P17=-3.6155874118732743, P18=-3.001806480565366, P19=-7.079894258742959, P20=-6.193780934200266, P21=-5.380762809137819, P22=-2.269119786752689, P23=-1.1444793863242662, P24=0.03108740007630868, P25=1.6964155978682685, P26=2.7143539042873597, P27=1.097307701670859, P28=-0.666156577133459, P29=-1.6389910281142324, P30=-1.4483926655437365, P31=0.7360519680705626, P32=-1.400376239158748, P33=1.7238525258663926, P34=-1.0665706716023369, P35=0.7807195371294661, P36=-2.054846380515178, P37=0.2844017074962113, P38=0.02191545390186786, P39=1.1440075187825511, P40=0.7041772000914589, P41=0.6459959279597611, P42=0.6317889518479644, P43=0.6998825438999052, P44=0.6751642754922746, P45=0.6702750385541574, P46=0.6559377946392358, P47=0.6030362616093861, P48=0.6099170182962445, P49=0.6278495231515453, P50=0.6502979882265212, P51=0.6721141578426704, P52=2.3758442955655562e-05, P53=1.3304647165483055e-05, P54=1.8190622358445365e-05, P55=2.4645910077493074e-05, P56=0.00015352350643987105, P57=0.0009482804323181401, P58=0.0018989566277468413, P59=0.004937162327043855, P60=0.021663670373814746, P61=0.05869303316956101, P62=0.07667526918484238, P63=0.03385152161382057, P64=0.005696007295341011, P65=0.0031246484385099227, P66=0.005507669000295779, P67=0.008962830800950795, P68=0.007995900130653134, P69=0.023144882623943677, P70=0.07118903180942923, P71=0.07145235189198891, P72=0.06491008402137198, P73=0.08890116362429833, P74=0.061874947712138285, P75=0.024360522718677353, P76=0.018072508249062452, P77=0.01788920441155362, P78=0.01673169862909756, P79=0.021877587937949677, P80=0.05862350331645688, P81=0.09167156634662059, P82=0.10934406490534879, P83=0.10607969482694223, P84=0.163249778242935, P85=0.15269715787876553, P86=0.03487723420773904, P87=0.03506317582806554, P88=0.06756748502867484, P89=0.046680680643187256, P90=0.061679369034637575, P91=0.07628610190364465, P92=0.16231027055292815, P93=0.2869834437334522, P94=0.21334538401571188, P95=0.07718757962033534, P96=0.014535680804113268, P97=0.01955421340389303, P98=0.05527492113209978, P99=0.1444676628205003, P100=0.07073249096537103, P101=0.02401674038315814, P102=0.004858454198892043, P103=0.003912716123238058, P104=0.008372885686491475, P105=0.030621547689218286, P106=0.050323372884578735, P107=0.01291191485958203, P108=0.005703608788103045, P109=0.029706203599157238, P110=0.04360554330192516, P111=0.0726998944690325, P112=0.022486554082994107, P113=0.018020950651763772, P114=0.009527540351588128, P115=0.006823920533531573, P116=0.007848499158087916, P117=0.008139593197940193, P118=0.003621978040189534, P119=0.0032722300340428468, P120=0.001414843387624601, P121=0.004006712315173338, P122=0.0040796606920882215, P123=0.0018543198783016615, P124=0.00593534833321798, P125=0.007899651391612873, P126=0.004841593617749084, P127=0.002127322262261514, P128=0.002379665404671196, P129=0.003919895799554534, P130=0.004551451359313287, P131=0.00442294483646628, P132=0.003519022422354618, P133=0.0026187754928583057, P134=0.004252034677462627, P135=0.003054840452310899, P136=0.003834524115548157, P137=0.0020711609374670617, P138=0.0013797365443248722, P139=0.0013386171806346304, P140=0.0010524739345594081, P141=0.0010275564604871966, P142=0.0008777407087944032, P143=0.0006336558285910218, P144=0.000881319403940088, P145=0.0012138287925307302, P146=0.0013618226614807484, P147=0.0008330760049695028, P148=0.000834302061567137, P149=0.0008289528536638286, P150=0.0011470606127646581, P151=0.0010128616952100268, P152=0.0010489851217133789, P153=0.0011027897841295784, P154=0.0006728861922884212, P155=0.00044468706516819775, P156=0.00043159437333875665, P157=0.0003169842797412937, P158=0.00041006792817604933, P159=0.0005810218986462169, P160=0.0009061540964292486, P161=0.0010633535049614207, P162=0.0010707472263007977, P163=0.0011623018825620145, P164=0.001054771237645198, P165=0.0006464049545810894, P166=0.0004763507617236492, P167=0.0004606175690047133, P168=0.0007492724226869662, P169=0.001247995616185178, P170=0.0009461831446899265, P171=0.0006918620971520868, P172=0.0006603966805070351, P173=0.0006033579156909767, P174=0.0005294111392396161, P175=0.0006572441797885818, P176=0.000833647774385257, P177=0.0010905345658144669, P178=0.00043489515768152244, P179=3.69149286296678e-05),Row( P0=-538.833420742108, P1=59.85522719938379, P2=-21.836821522830817, P3=-4.098368054944471, P4=-22.195242364951216, P5=-23.913899576995917, P6=-14.981816101937122, P7=-4.801985711078284, P8=-10.870596788768852, P9=-14.768687218906562, P10=-6.321179800820745, P11=-9.701323426963901, P12=-8.740327882959942, P13=-8.969416199019392, P14=-7.73037950235124, P15=-1.6941313634079707, P16=-4.081139036062478, P17=-0.20328442309981842, P18=-5.569571735050365, P19=-3.404399420388798, P20=-7.42384696121629, P21=-1.262185295603046, P22=2.6768753589070595, P23=3.5557587821025742, P24=5.549986386125981, P25=8.757106239371973, P26=5.0065972173633755, P27=1.521180830240257, P28=0.4031797486106854, P29=1.4042581449945195, P30=-3.6005097341162746, P31=0.5611149884128082, P32=-1.17993596608786, P33=0.996993681703086, P34=-3.207942912954396, P35=-0.5477856684605494, P36=-3.4398173842319615, P37=-1.8218658485603252, P38=-3.1179706145970356, P39=-1.0058455815056375, P40=0.6773140655556749, P41=0.6314194094283755, P42=0.5722585678979244, P43=0.6227703603808067, P44=0.6669890524326394, P45=0.6741796435644725, P46=0.6528092899653839, P47=0.6058320257339872, P48=0.5569918491023629, P49=0.5642030857822158, P50=0.6602185907988801, P51=0.7056855821484915, P52=6.377569928419497e-05, P53=1.1099416426209176e-05, P54=2.6678029606105312e-05, P55=6.699959329562636e-05, P56=0.00016233138762768815, P57=0.0003782189073765833, P58=0.00041736982809566285, P59=0.0022290216318338004, P60=0.01592459421501583, P61=0.09049288462018075, P62=0.16301982211139243, P63=0.11466070059300695, P64=0.020755021861204676, P65=0.004060114522121879, P66=0.0028628199044252, P67=0.005943808338317978, P68=0.00886545254267398, P69=0.008923036540848707, P70=0.016974277295330534, P71=0.062353738987978524, P72=0.1109997739526668, P73=0.10892030757294163, P74=0.0908564150870245, P75=0.059759598379108915, P76=0.029215436874568377, P77=0.018794899266769558, P78=0.016700573800555986, P79=0.019885009830671325, P80=0.028487479261871573, P81=0.042725337266960234, P82=0.06450888113543492, P83=0.06129978905279375, P84=0.07053702880844705, P85=0.10685600715790176, P86=0.05781784575426497, P87=0.021078836500312798, P88=0.016809084038739187, P89=0.015501961453903312, P90=0.034920560710134975, P91=0.03898775104602484, P92=0.0397687067175888, P93=0.011412437242826045, P94=0.0971949552345904, P95=0.1517275517357649, P96=0.03017697583832619, P97=0.035305636465950384, P98=0.02600693383435317, P99=0.02051255038562376, P100=0.0047367570488856875, P101=0.037424078968007285, P102=0.036904208548251334, P103=0.012181463811264193, P104=0.0028966064498877372, P105=0.0018706012211309206, P106=0.01402340621021207, P107=0.080246277136083, P108=0.021296670794181446, P109=0.0032458951529482196, P110=0.003370476181505863, P111=0.019835390346059262, P112=0.029918185703034124, P113=0.01646098063350664, P114=0.00299978386525507, P115=0.003483075795292523, P116=0.0027457070985030265, P117=0.0020617644646020563, P118=0.0030466076665154294, P119=0.003123615294623742, P120=0.000991332846736415, P121=0.0020062044982303642, P122=0.0017056634373525572, P123=0.0012320084122087807, P124=0.0018260033669092665, P125=0.0015764589810586846, P126=0.0038009812813006223, P127=0.004600947352529754, P128=0.0032774203146929016, P129=0.0024508383074618727, P130=0.002087602286095842, P131=0.0016257126458293595, P132=0.001521447420315071, P133=0.0021792111594235494, P134=0.0015961489816853566, P135=0.0015178981743275552, P136=0.0014570423382384006, P137=0.0007242646071904258, P138=0.0005019951182356798, P139=0.0005117525136329323, P140=0.00033981750854384854, P141=0.00048270199256257443, P142=0.0004121137492949817, P143=0.0003357979251774088, P144=0.00037912915829328876, P145=0.0002983245941046874, P146=0.0003283509187333487, P147=0.0003100430310540427, P148=0.0002264159498547856, P149=0.00028069429651768167, P150=0.00029040591114894273, P151=0.0003296017844023262, P152=0.00033354933133776467, P153=0.0002742210861940126, P154=0.00024763322284464435, P155=0.00022767335753941034, P156=0.0004652113394015042, P157=0.0006196608349878026, P158=0.0006619260503650277, P159=0.000547201967121073, P160=0.0003748458564846169, P161=0.0003793539355037115, P162=0.0005289174731309335, P163=0.00019251076064195362, P164=0.0002145069936611136, P165=0.00026112489535088523, P166=0.00047382335679847434, P167=0.00034445364791967623, P168=0.0002274320073726496, P169=0.0001830032532862954, P170=0.00020408056409694687, P171=0.0001430943852824466, P172=0.00020634444496709785, P173=0.0002795806922970335, P174=0.00037206201623208007, P175=0.0002501119735725297, P176=0.0001435647864497126, P177=0.00017462717535300933, P178=0.00014029116212463762, P179=2.3288905344842648e-05),Row( P0=-550.0590944404247, P1=58.28165753414515, P2=-16.670174192280964, P3=0.849177650452818, P4=-7.520273264702217, P5=-10.892669171436768, P6=-2.574408561061083, P7=10.000932290730912, P8=-4.611254616542598, P9=-1.217879375382767, P10=-2.6833823208923953, P11=-8.124550436646269, P12=6.0019635216001985, P13=-10.927810199981176, P14=-0.49347945330538323, P15=1.4545197462193658, P16=-2.4166941617723285, P17=3.5354959337225407, P18=-5.984749828438784, P19=-0.3856831595141829, P20=-5.289332772707416, P21=-3.0671333973693073, P22=1.2869556081571678, P23=-0.5246216577179755, P24=7.548833556283179, P25=11.998165021313259, P26=9.32486694313078, P27=12.216749294949643, P28=7.549624195055588, P29=5.000052345658643, P30=-0.23388690023168413, P31=4.167817442451833, P32=3.7219587982479196, P33=7.498278889257754, P34=6.674473935507312, P35=2.0902431533676307, P36=-1.2541856089311592, P37=-0.6654681547767692, P38=-0.46115979844948973, P39=5.234117635186594, P40=0.5854633477216283, P41=0.5536947033202188, P42=0.620842806480907, P43=0.7569594815704509, P44=0.7412471006369667, P45=0.6580350520004226, P46=0.611204258402456, P47=0.6079580425662011, P48=0.6016170626081305, P49=0.6488740771652645, P50=0.725431498410212, P51=0.6722863567749188, P52=0.028314061272791874, P53=0.0027495847320548423, P54=0.000256333972410659, P55=3.9695143313811785e-05, P56=8.05003071042384e-05, P57=0.0008098936918748065, P58=0.0069887993122435136, P59=0.0235406437278542, P60=0.08030074645317561, P61=0.2941093899340127, P62=0.21488401784345204, P63=0.014986188673365826, P64=0.0008162630488834416, P65=0.0010988586904637958, P66=0.006591041755341316, P67=0.01780366513251468, P68=0.031245384403330478, P69=0.053684533183410335, P70=0.06254717830191772, P71=0.13403760707967394, P72=0.18096298772097555, P73=0.0757952023604057, P74=0.007681620866182543, P75=0.005490026108462475, P76=0.009606937321520684, P77=0.03647795677792901, P78=0.0515983744167859, P79=0.05780487825099346, P80=0.07400624425114162, P81=0.14269880901748674, P82=0.21206415427756425, P83=0.13714481561646646, P84=0.05961918176388225, P85=0.031034730414655896, P86=0.023390842115016346, P87=0.018304840587005287, P88=0.019455531182106144, P89=0.04405931643321582, P90=0.12280783088947896, P91=0.14709910177889837, P92=0.04828686614359434, P93=0.011725976648572055, P94=0.006505825937913319, P95=0.008400261386644813, P96=0.0924965636148329, P97=0.14499079137679338, P98=0.041853084036633396, P99=0.00806229967355981, P100=0.0031656551864526422, P101=0.007546171588709459, P102=0.066995816150501, P103=0.03070365665899451, P104=0.00541214989342101, P105=0.0013174968500927959, P106=0.0037528965711150905, P107=0.017270707909091312, P108=0.005156042953604459, P109=0.001046813290640943, P110=0.011103855918184122, P111=0.06392416281148205, P112=0.015730383552093004, P113=0.0032494655761722453, P114=0.006791718368101982, P115=0.019472035226891938, P116=0.0023873866930752374, P117=0.004644013960531728, P118=0.02169527611286453, P119=0.005170931050595277, P120=0.0025916048667933445, P121=0.008364189161114975, P122=0.0039548462522191125, P123=0.001927118845219317, P124=0.004742729351212647, P125=0.002944602293329032, P126=0.004157144167337787, P127=0.0028015525423325215, P128=0.0021300415529073675, P129=0.0011965834173810527, P130=0.001503993014935918, P131=0.0007972633606401938, P132=0.001531029660861225, P133=0.0014167075616243572, P134=0.0016556730089039523, P135=0.0022671670095393225, P136=0.0031238506388563923, P137=0.0011652281439592726, P138=0.0008555072385508577, P139=0.000505323506589169, P140=0.0004269620843615733, P141=0.0005865719591563785, P142=0.0004908624554771875, P143=0.00039657389254741103, P144=0.00039651793329944666, P145=0.00030847630619650263, P146=0.00013295799323064178, P147=0.0001352376710384641, P148=8.669023699168035e-05, P149=3.8920173054051287e-05, P150=2.6368587934983107e-05, P151=3.886213911425689e-05, P152=0.0001193254279189426, P153=0.0002480671347697779, P154=0.0003564747856388037, P155=0.0003882786748288734, P156=0.0006449919309298942, P157=0.0009590539883340456, P158=0.0014395046034941176, P159=0.0016885398939194144, P160=0.001569408965707998, P161=0.0015965199331018538, P162=0.001513771364385893, P163=0.0010514250584138984, P164=0.000831378040148741, P165=0.0007327334362553967, P166=0.00078757908591871, P167=0.000562608541099486, P168=0.00026805286157496227, P169=0.00017356417157718136, P170=0.00012225840006195324, P171=0.00016258767015177664, P172=0.00022933394488205362, P173=0.000230621832759488, P174=0.0002683489219687481, P175=0.0002092795784997624, P176=0.00016290725731452646, P177=0.0001878049491932543, P178=0.00012450649964722009, P179=1.4345692591032427e-05),Row( P0=-551.7583634130549, P1=50.483333991377336, P2=-16.99184529724486, P3=-4.916864290052423, P4=-20.50031106927231, P5=-19.64676088956941, P6=-14.27535899040438, P7=-2.443820991728084, P8=-6.9446464490965045, P9=-11.034661521978657, P10=-7.208592729067661, P11=-9.128290258998737, P12=-4.682388866888216, P13=-8.254158199865598, P14=-6.593445726024718, P15=-0.5484881821567675, P16=-4.454134253663809, P17=-1.921152106678008, P18=-4.68555205205061, P19=-3.5647439252712543, P20=-5.00878098061185, P21=-0.0351721656295328, P22=6.249292398411912, P23=6.681000633976234, P24=7.62450848108157, P25=9.408805806758638, P26=3.871750859894326, P27=2.481620745592684, P28=1.693615376231959, P29=2.320325603346375, P30=0.26345409668462066, P31=4.612618533651059, P32=1.3016645031227958, P33=0.5230951852594902, P34=-3.6630641020371666, P35=-2.0603543569922227, P36=-2.2706346135390527, P37=0.38722683698891613, P38=-0.688122088959736, P39=0.3739641037377341, P40=0.6830843926761472, P41=0.6776949660670963, P42=0.6312110798348242, P43=0.6311242904979516, P44=0.6732124093460897, P45=0.7631313777691036, P46=0.7401335493488015, P47=0.684173902682407, P48=0.606336566408075, P49=0.6175820636994053, P50=0.6604184365571113, P51=0.650354828995764, P52=0.0004604123642306312, P53=0.00011182078537144477, P54=1.8905441374907136e-05, P55=3.3987685767345225e-05, P56=5.8705346688301054e-05, P57=6.452509162338708e-05, P58=0.00036773365538363893, P59=0.004730748206362552, P60=0.02236636788952747, P61=0.07552387483189026, P62=0.16434159681599855, P63=0.12700840710593445, P64=0.02758137070026635, P65=0.005209221330705297, P66=0.0029572595817628825, P67=0.005854505407466795, P68=0.015683948730503255, P69=0.025437061039082377, P70=0.04160112798279287, P71=0.060916540257729135, P72=0.22322206795892766, P73=0.25038786074582164, P74=0.13657706063451922, P75=0.1318698928102623, P76=0.08603933370905972, P77=0.049411371622808044, P78=0.02969175141716273, P79=0.023420184764115535, P80=0.024276620371921367, P81=0.029407219955012387, P82=0.06591353801681937, P83=0.16831822766094554, P84=0.2606395856412061, P85=0.21026721039509652, P86=0.11831234388194409, P87=0.05894520237627694, P88=0.03670135229958805, P89=0.014854486994420152, P90=0.01576885135259402, P91=0.03233970719535237, P92=0.07582971653581876, P93=0.0694382284341602, P94=0.10297968366183295, P95=0.1633059217601307, P96=0.07433910702803527, P97=0.03704933557390233, P98=0.028490251102571366, P99=0.022931848179370135, P100=0.02547346695399422, P101=0.030681865400153902, P102=0.015324660149003218, P103=0.004799846722114083, P104=0.0012587536144757292, P105=0.0062127730154448065, P106=0.015616728276514315, P107=0.028913867636124576, P108=0.01854082525858879, P109=0.0030545123845381657, P110=0.013574151402902992, P111=0.043468732297326236, P112=0.030147439879629902, P113=0.008250404840939996, P114=0.00754733978027037, P115=0.005529956865736422, P116=0.0033650595505913343, P117=0.0036851490167745405, P118=0.002332985056295496, P119=0.001861810159197272, P120=0.002361365265127931, P121=0.0027081023116734375, P122=0.002984161888360192, P123=0.006711572498390208, P124=0.0074552007625348846, P125=0.003048623268400849, P126=0.004023216299421252, P127=0.004571193546849628, P128=0.0016137947008982502, P129=0.0018157778702896428, P130=0.0017038424895775425, P131=0.0013615671825313862, P132=0.001205023220898048, P133=0.002566198473557178, P134=0.0031065730378712305, P135=0.0023657874013794772, P136=0.001173194416432794, P137=0.0004653101271989898, P138=0.0009355163594632216, P139=0.0011310556033489827, P140=0.001331554853463091, P141=0.0017789609018786774, P142=0.002533263185939463, P143=0.0011659667725490894, P144=0.0006191026018905223, P145=0.0004128623151598919, P146=0.00035322360158510265, P147=0.000313081430588589, P148=0.0003656610052393704, P149=0.00038406828401279304, P150=0.00027816661281313026, P151=0.0004615833010327072, P152=0.0005608105377037318, P153=0.00038371185144552, P154=0.00040439113980714176, P155=0.0005613934466443419, P156=0.0006599126723296347, P157=0.0008889710615905338, P158=0.000801542357220149, P159=0.0006453364908757829, P160=0.00042375131078461076, P161=0.0005953093556694468, P162=0.0004017963446188437, P163=0.0003354493493313476, P164=0.0006912725833722349, P165=0.0006117806940235678, P166=0.00045425097802248554, P167=0.0005663238100034919, P168=0.0004994494123103996, P169=0.0005131819758349162, P170=0.0006258905291249279, P171=0.0005002191652924307, P172=0.0008478002634424333, P173=0.0004368536033083213, P174=0.0003187915496246196, P175=0.00017835850284540414, P176=0.00021679610344237844, P177=0.0001767448727447749, P178=0.00011768476445394935, P179=2.2147612046694708e-05),Row( P0=-461.2593947052097, P1=37.49956523238904, P2=-27.995147468604944, P3=-13.302447993878383, P4=-15.909037554483428, P5=-11.861915188063659, P6=-15.617248243448705, P7=-0.9586195348599194, P8=-9.411365732810806, P9=-8.80054178956611, P10=-7.855538282960982, P11=-10.08360654299201, P12=7.771898318398332, P13=1.4344272326365823, P14=13.668566153804017, P15=16.99808608145571, P16=8.127878959146301, P17=8.00586864390621, P18=-1.928764555259187, P19=-1.1298644891429117, P20=-2.434460383969306, P21=4.031071220596262, P22=2.618733974686261, P23=-1.7060419005565222, P24=-4.740364929014815, P25=-0.6893785665509536, P26=0.6958258159492305, P27=8.63722327019416, P28=3.57564138198779, P29=2.2585813023122068, P30=-0.04874947461436608, P31=2.496011965155774, P32=1.8974322414178693, P33=0.9857921604268706, P34=-3.628490394106646, P35=-0.9540291294903069, P36=3.5056564110794035, P37=4.8822974340443395, P38=0.8241885519233422, P39=0.08889921596187003, P40=0.6025974236016074, P41=0.6828365697586082, P42=0.7215769008593583, P43=0.6338122161885832, P44=0.62109012344498, P45=0.6117981301152166, P46=0.5523693274354677, P47=0.534996354112233, P48=0.6068144399423087, P49=0.6366969243810994, P50=0.5602018354726067, P51=0.5517759144748301, P52=0.004191107893002354, P53=0.0008394842022542688, P54=0.00024082866232941777, P55=4.728509993961544e-05, P56=9.412580202229145e-05, P57=0.0006748296402716923, P58=0.0010806522437039696, P59=0.0006606287541169299, P60=0.0007925540802854122, P61=0.002503179729980949, P62=0.014478047369899416, P63=0.03311317942157183, P64=0.0476603411545055, P65=0.1651470029126957, P66=0.23220044416976823, P67=0.24876753218286668, P68=0.60568906283961, P69=1.4011382280611955, P70=1.3033439193565237, P71=0.45672136582737666, P72=0.20704030991690825, P73=0.26895966242385005, P74=0.11718037582198057, P75=0.012113411809596132, P76=0.0063219652551777805, P77=0.0138305979627766, P78=0.08616051264785883, P79=0.1960396963965594, P80=0.09235874905412926, P81=0.051032647474157214, P82=0.040205966840323064, P83=0.0671397096950303, P84=0.19706368435880908, P85=0.5603114905613482, P86=1.9145327939082073, P87=3.5482782258726147, P88=3.545232403900717, P89=0.6683470096249843, P90=0.15939577809838076, P91=0.12115402696555745, P92=0.28772982258797253, P93=0.46051723010598017, P94=0.42755936991453575, P95=0.17387040074456547, P96=0.03606226935954744, P97=0.06311960455437952, P98=0.5763031365602735, P99=2.800827355511298, P100=1.3797004226882315, P101=0.35759914488360844, P102=0.029900756290378628, P103=0.017339236249624374, P104=0.007546792471299304, P105=0.008367481851577221, P106=0.017647499530909695, P107=0.14183032848174407, P108=0.2256245251358896, P109=0.10817002476082932, P110=0.05444331136118486, P111=0.023276865538556887, P112=0.007490171765301265, P113=0.006943708766729727, P114=0.14269690118908346, P115=0.1976309915975327, P116=0.032098478003754906, P117=0.02737963551502841, P118=0.030278281285818994, P119=0.03571260437515855, P120=0.014138983944581117, P121=0.0452056630466668, P122=0.060804370707581844, P123=0.01669525160003934, P124=0.00499345354176826, P125=0.008485632139400666, P126=0.020139344498278835, P127=0.020929875344655277, P128=0.003217506602555169, P129=0.003486247014984056, P130=0.015510274860009204, P131=0.0062651445730242655, P132=0.001832985209060088, P133=0.005460784347964892, P134=0.010985197980217765, P135=0.005817211795888781, P136=0.00764152943496326, P137=0.005811670286952083, P138=0.0027212052178854263, P139=0.001771955470490198, P140=0.0018552304895388817, P141=0.0011464803971815332, P142=0.00081237395084035, P143=0.0008907739128970845, P144=0.0009300132909704369, P145=0.0019802395048864603, P146=0.00907955514389951, P147=0.001906288562785509, P148=0.007948287166216213, P149=0.002497629487068497, P150=0.0022266674241340004, P151=0.0010733922837345416, P152=0.0018884711539404401, P153=0.0018045373182346551, P154=0.0022511178316111237, P155=0.0056947526710687145, P156=0.005240027150719899, P157=0.007023918223121879, P158=0.005056288369641032, P159=0.004539983273699755, P160=0.0021437802064365667, P161=0.0011282288330701719, P162=0.0009399264026096686, P163=0.0009131275225031099, P164=0.0004623411941541686, P165=0.000931858755893132, P166=0.0006850535833917108, P167=0.0005259457118592775, P168=0.00030153343756559805, P169=0.0003856538221990312, P170=0.0003990654758449574, P171=0.0005372790430308981, P172=0.00034975024206269174, P173=0.00046167283291562346, P174=0.000699230004712451, P175=0.000724824799743345, P176=0.0010486846398340824, P177=0.0010626943981926387, P178=0.0004852933175733382, P179=4.937271246231874e-05),Row( P0=-567.1570916824945, P1=62.56281062338139, P2=-19.51734189115149, P3=2.2859758912352, P4=-19.616939220978757, P5=-17.01698138104275, P6=-4.352896523830452, P7=-5.456346040515744, P8=-13.224079607352884, P9=-11.19589720054043, P10=-9.303507575211762, P11=-9.111799035831801, P12=-6.7852798002686106, P13=-11.117165371182388, P14=-6.425056488469739, P15=-4.724275862383485, P16=-4.713336329118714, P17=-3.3105595335940996, P18=-3.286150165737152, P19=-5.842932359832184, P20=-4.922209660859218, P21=-4.950659564342655, P22=-3.2536800660266283, P23=-2.5073291253313674, P24=-2.867555027086515, P25=0.16306205896879652, P26=-2.8655779699444928, P27=2.014439755658436, P28=-2.266952444175463, P29=1.1545491456636885, P30=-0.008870880863940337, P31=0.7924610359640318, P32=-2.573211061377437, P33=0.643896835342097, P34=-1.5472773725870714, P35=-0.0870468894100098, P36=-3.5207568486544982, P37=0.5100715956068225, P38=-1.2332543182444171, P39=-0.05750251112610974, P40=0.7359242444177124, P41=0.703140308419666, P42=0.6557957639512804, P43=0.6672206513021283, P44=0.7140190978487868, P45=0.7418256482712675, P46=0.7249596735387495, P47=0.7216973655198832, P48=0.7218982951017747, P49=0.7094622033426525, P50=0.7334864557290348, P51=0.7420679032815126, P52=9.197370281857737e-06, P53=6.420718904396193e-06, P54=3.220115255734218e-05, P55=0.0003200546580092701, P56=0.0007324255913494923, P57=0.001247746209407118, P58=0.0013471905208520278, P59=0.003541218764270982, P60=0.011289804205452076, P61=0.023875763144416032, P62=0.09387204749008886, P63=0.1542784496399593, P64=0.030684025558499577, P65=0.002900065584853944, P66=0.0038092447283433984, P67=0.005815943145791581, P68=0.012769940478702906, P69=0.011514946860531323, P70=0.012119222464124984, P71=0.011204292779865028, P72=0.009400038726486376, P73=0.012660106469653098, P74=0.034304937863930976, P75=0.0670669085631136, P76=0.030769203443973714, P77=0.007387190977897451, P78=0.006940858495974084, P79=0.010765763190072044, P80=0.014557539057614593, P81=0.019214743839141773, P82=0.020687978719755036, P83=0.009689771498379088, P84=0.01096602947999372, P85=0.016762193155424616, P86=0.017139693935820485, P87=0.01590572254676842, P88=0.0078054543694281945, P89=0.004073701385975487, P90=0.004874904488629886, P91=0.007139555409639394, P92=0.01437745153788435, P93=0.012232941073886986, P94=0.022327143734849415, P95=0.023638060661993296, P96=0.008188565905283815, P97=0.010020156500251983, P98=0.0164584406025564, P99=0.009445935924558321, P100=0.014531190104848907, P101=0.022987630210125082, P102=0.015519076677512362, P103=0.004674368535729051, P104=0.003328710160465657, P105=0.007882186669365023, P106=0.014969101807122685, P107=0.012991313154932875, P108=0.0033188777712874983, P109=0.0018078973770265768, P110=0.005248932052320132, P111=0.009988699583675952, P112=0.005833586507644953, P113=0.002890310412232408, P114=0.0008928644756860489, P115=0.0003507554880861075, P116=0.0008897226203966331, P117=0.0015989080391202008, P118=0.0005278383921975531, P119=0.0004447207602994603, P120=0.0005223639227567805, P121=0.0008343731355497477, P122=0.0005355527735964538, P123=0.0004039517539068199, P124=0.0007140455104325803, P125=0.0009407674574818125, P126=0.0021933864105008064, P127=0.00414175113838529, P128=0.001441357803814345, P129=0.0011459891947985532, P130=0.0011684037855894582, P131=0.0008430929550869246, P132=0.002240765467199993, P133=0.002163046531690924, P134=0.003180067084156512, P135=0.002251165017005378, P136=0.0027058250813900018, P137=0.0023200103669065615, P138=0.0018093356908355587, P139=0.001357864048199742, P140=0.0009099549811821061, P141=0.0009113414747646381, P142=0.0004731703078649804, P143=0.00036358618976173155, P144=0.00045858657702676847, P145=0.0005343113370926243, P146=0.00034246099036625694, P147=0.00030282033798606137, P148=0.00014893115112531204, P149=0.0001033033979613031, P150=0.00013323242631132663, P151=0.0001794610935608508, P152=0.00014977226842415056, P153=0.0001612137378679821, P154=0.0002422292675655533, P155=0.0003764285708868213, P156=0.0002488928834935777, P157=0.00024407803537426396, P158=0.00019424864181001572, P159=0.00014932013048971226, P160=0.00012948798266107481, P161=0.00010907851877970926, P162=0.00012801899649284555, P163=0.00018120154594224654, P164=0.00021173188688696613, P165=0.00024540948887788835, P166=0.00019240158792687708, P167=0.00011368994887965734, P168=8.572512411193539e-05, P169=7.787654107722266e-05, P170=6.456533383813434e-05, P171=6.653789303150324e-05, P172=8.909610613630721e-05, P173=9.068560934333741e-05, P174=9.863574190219996e-05, P175=7.462652060371496e-05, P176=8.46947243455957e-05, P177=0.00013186346664592667, P178=6.964770950578445e-05, P179=9.857916928387651e-06),Row( P0=-551.8982563817843, P1=54.132060308093166, P2=-17.009935466964105, P3=-2.363575989903059, P4=-9.316607380921296, P5=-11.160566507089575, P6=-6.177315210901241, P7=8.544313830493438, P8=-6.752702763791135, P9=-5.740485440929473, P10=-2.2047960821441377, P11=-8.680951901993827, P12=4.4778718105693045, P13=-12.894279154325936, P14=-0.4597347242657089, P15=0.6547209688033302, P16=-3.5928764279668073, P17=2.8342071319043747, P18=-8.097884534845477, P19=-0.611858713069041, P20=-2.7169717654223815, P21=0.27073658695850494, P22=5.835811939308053, P23=5.109656788280206, P24=9.062356516667077, P25=13.378290101624865, P26=5.131769433650936, P27=7.921820746116269, P28=3.6729934212420843, P29=2.703670299920656, P30=2.475249178371108, P31=5.645223092197596, P32=3.906627578883943, P33=6.175498002644582, P34=3.268201106912611, P35=1.034083076601812, P36=0.612164120959081, P37=2.5795626331020722, P38=1.490905978864053, P39=2.987150184793103, P40=0.6557201139674756, P41=0.6342666988316752, P42=0.545850992392272, P43=0.5774263953530593, P44=0.6523937422358709, P45=0.7105311465227298, P46=0.7223085944130764, P47=0.6190153393150338, P48=0.6191412176929056, P49=0.619462857132129, P50=0.6440202807296251, P51=0.6539082083600897, P52=0.005044796114555727, P53=0.00029979437350218923, P54=3.674983006486759e-05, P55=1.6598799044618235e-05, P56=3.258326873604011e-05, P57=0.0003630753891997095, P58=0.006315507358667117, P59=0.020428816367857927, P60=0.041758008902497694, P61=0.12961935909700292, P62=0.22503299408413815, P63=0.1059291747272903, P64=0.004462767361898257, P65=0.0013833761472598028, P66=0.0070281189660336375, P67=0.015610722422969804, P68=0.025108859450606983, P69=0.034474651542008246, P70=0.02959209154802745, P71=0.061527153643584125, P72=0.08708111522035515, P73=0.1591987990478469, P74=0.17028658988894685, P75=0.02952776644056425, P76=0.010098899176186803, P77=0.025911011459720572, P78=0.03247271607761919, P79=0.024491365800222088, P80=0.020816213039486844, P81=0.05904320863036703, P82=0.09610988835277211, P83=0.09648634729315278, P84=0.18324251325124324, P85=0.15692935560766375, P86=0.037383210841872315, P87=0.012307305166703775, P88=0.011135871500521515, P89=0.023798336743221717, P90=0.09881032824152772, P91=0.11819904524497186, P92=0.09301420648224422, P93=0.06232404429929111, P94=0.00891950762912319, P95=0.003071448502888451, P96=0.037250636293256346, P97=0.11296952762081616, P98=0.06040400427056567, P99=0.03239334792483659, P100=0.012848046036023771, P101=0.004138003699327287, P102=0.043019216555133126, P103=0.01953053678100131, P104=0.003316144308792766, P105=0.008703900054127408, P106=0.005267582195761768, P107=0.015454228362999653, P108=0.003901067607906151, P109=0.002690169854775724, P110=0.01181380896026868, P111=0.038872512666807454, P112=0.011469643237574153, P113=0.001477433150857281, P114=0.0051514507811591835, P115=0.014819192099910672, P116=0.001981738393283749, P117=0.0020985661326098992, P118=0.010952815602336272, P119=0.004823921499099358, P120=0.0015539328389692067, P121=0.0026249211407028622, P122=0.0046829328251113194, P123=0.0013480924088535387, P124=0.0021934031047755953, P125=0.004188644369806867, P126=0.002022869457307964, P127=0.0037027846527755708, P128=0.0033993147504476507, P129=0.0006947857914010006, P130=0.00215887083277089, P131=0.0006841104164442161, P132=0.001534095042003872, P133=0.0014498677323009344, P134=0.001721284762043945, P135=0.0030306674280614647, P136=0.0031080167441596904, P137=0.0014379543778481401, P138=0.0006064406993557254, P139=0.00039534635816645107, P140=0.0002941424279966383, P141=0.0003793197591857619, P142=0.00043738295693959787, P143=0.0005168170295549573, P144=0.0005199397700660552, P145=0.00020378862838639356, P146=9.09793647734208e-05, P147=7.396541989305975e-05, P148=4.830568681265671e-05, P149=3.156834624536967e-05, P150=3.0547841618563914e-05, P151=5.254719394895917e-05, P152=0.0001294921918876402, P153=0.0003126879070485172, P154=0.0004159567436552147, P155=0.0004968809125178735, P156=0.0009321042055230943, P157=0.0013229439041415414, P158=0.001443867942903006, P159=0.0018332672865266997, P160=0.0024658111809948663, P161=0.00205577189495874, P162=0.0014574363316798991, P163=0.0010675956213179495, P164=0.001084965844630805, P165=0.0015679796715849681, P166=0.0014800563042460346, P167=0.0005996936224765005, P168=0.0003563437152779614, P169=0.00038280246346398075, P170=0.00027889943656172794, P171=0.0004983202305926302, P172=0.0004820982395353061, P173=0.00047850333646830496, P174=0.00039221377183671437, P175=0.0003116615275614219, P176=0.00029007379331940264, P177=0.0004129603607556918, P178=0.0002524704359108882, P179=2.9986553139491354e-05),Row( P0=-550.5992580093663, P1=41.77267528959492, P2=-19.29555965537005, P3=-7.245891665476632, P4=-19.0597725397455, P5=-21.56138325357923, P6=-17.123085293885012, P7=-1.71560436216455, P8=-12.44610679122788, P9=-8.493948483079908, P10=-6.336025734030775, P11=-10.198546976071277, P12=-2.102281056293213, P13=-9.821083753920183, P14=-3.3650138309488464, P15=-3.359077828099832, P16=-2.988721910719553, P17=1.4831550153041833, P18=-0.524601637827771, P19=3.3075809339289957, P20=0.35680136058635226, P21=1.1851570955720594, P22=1.8812275423747078, P23=-1.7308610872637598, P24=-2.470741203405877, P25=-0.8259486522828999, P26=-1.9105511751760575, P27=2.34094546649606, P28=0.7575335032646685, P29=1.2338250135128457, P30=-2.072492972687548, P31=-0.6399147705664161, P32=-1.6028424109219486, P33=2.3098154397779957, P34=1.2972149176731835, P35=0.4866355927675515, P36=-0.42238852719014003, P37=0.9635830572484543, P38=1.7195288093593992, P39=1.8088977067570713, P40=0.5505541632911459, P41=0.5184376819413213, P42=0.5696904379644986, P43=0.5456630920543866, P44=0.5525354125555278, P45=0.5336510357061425, P46=0.5432459179575978, P47=0.5890807192065373, P48=0.5871740678248812, P49=0.618652226463663, P50=0.5957233416904179, P51=0.559369265509946, P52=1.9063352013265907e-06, P53=3.185000111267967e-06, P54=3.530880115121203e-06, P55=2.058092928175325e-06, P56=4.980129363686164e-05, P57=0.0005087006339701868, P58=0.0021058437500051735, P59=0.00510250723164905, P60=0.017330671240756854, P61=0.03293852451751476, P62=0.04208930912455573, P63=0.06135010554949169, P64=0.06317862196075935, P65=0.1325053718444106, P66=0.12170362291943622, P67=0.03585082199543927, P68=0.034344572845432186, P69=0.03675740058476772, P70=0.04581397435086663, P71=0.03486839021953226, P72=0.03232066987331606, P73=0.042329131736255424, P74=0.03311662718823337, P75=0.028340511690012914, P76=0.04693977695595867, P77=0.09629802094628907, P78=0.15581931768429944, P79=0.4731413961811704, P80=0.7006486597322743, P81=0.2869371333614961, P82=0.10131888490487699, P83=0.10195023931812522, P84=0.15536795585550323, P85=0.1942215716243027, P86=0.1036187054799325, P87=0.0884442559266531, P88=0.3151597035080655, P89=0.20217007654867566, P90=0.13198232839970214, P91=0.08358363944628464, P92=0.05620091811271941, P93=0.01706857177117728, P94=0.004701324480536918, P95=0.0064393681111678605, P96=0.06614194874637964, P97=0.3243803749948818, P98=0.19614434235445244, P99=0.059210641429929906, P100=0.004079380572937028, P101=0.0020136211437269454, P102=0.0024533451089067976, P103=0.0031005194617703042, P104=0.008456514112084087, P105=0.0070269742427205295, P106=0.009867630782200364, P107=0.008925616901388475, P108=0.001891435886192245, P109=0.004093888523338869, P110=0.007312863338606254, P111=0.026214569647397892, P112=0.014289253935990079, P113=0.0019506464147458797, P114=0.02896404086349893, P115=0.01808470574229479, P116=0.0018998182732668236, P117=0.006552471392974735, P118=0.02408904078306626, P119=0.022806303907115157, P120=0.005871482512480118, P121=0.003789181440958183, P122=0.024755859424075714, P123=0.008720302498772233, P124=0.0020029423617067567, P125=0.003950418200376055, P126=0.003269809985044182, P127=0.003739452699210679, P128=0.002690872521969183, P129=0.0019108408840254939, P130=0.002391863809657356, P131=0.002707638130563264, P132=0.0017313604435529545, P133=0.0040441365254933175, P134=0.003871413217010065, P135=0.002294654086859619, P136=0.0018248065644789462, P137=0.0007562620947364534, P138=0.00036182160223119153, P139=0.00036797820990306995, P140=0.00023353085298056928, P141=0.00044991583460725164, P142=0.00020860277837592622, P143=0.0003834131531809197, P144=0.000316079904499142, P145=0.0004625896885314813, P146=0.0003602888125644131, P147=0.00021381743198839264, P148=0.00019993608968739768, P149=9.885599337440639e-05, P150=6.600052813723982e-05, P151=5.456453944758616e-05, P152=9.565594895541207e-05, P153=0.000178363404687525, P154=0.0002858297262697978, P155=0.00034015553608257056, P156=0.00047388097749701306, P157=0.0006758910301380742, P158=0.0006473029716628393, P159=0.0006728014654661372, P160=0.0006810671094366408, P161=0.0009990534575311637, P162=0.0009623334011599784, P163=0.0008130869940461141, P164=0.0004010609013459893, P165=0.0004973276859812575, P166=0.000924514027698261, P167=0.0009606619353951565, P168=0.0005012444808338154, P169=0.00022995158618869824, P170=0.00010589158447910519, P171=6.353700483035622e-05, P172=0.00014449267958790168, P173=0.00017632107168894046, P174=0.00011628291379993514, P175=7.792328817039349e-05, P176=8.279290853540064e-05, P177=0.00011874868365714174, P178=6.13119351513629e-05, P179=8.574413921954683e-06),Row( P0=-594.4191804756135, P1=62.652775327619906, P2=-12.76615443879494, P3=-0.7100642090861248, P4=-16.140749057312973, P5=-17.480036759189886, P6=-12.87106125023986, P7=-0.7616642230958058, P8=-5.417919687482287, P9=-14.325231822717711, P10=-3.16193170431573, P11=-5.438330985449486, P12=-4.970350751590884, P13=-6.987689684503358, P14=-4.8317748071756705, P15=-1.3893323752611677, P16=-1.6204946402558562, P17=0.26276153468604335, P18=-4.348548688645642, P19=-0.6917226827018839, P20=-3.018322391575943, P21=0.5464718379089459, P22=1.2359733628745238, P23=1.8953111035461527, P24=0.8225675218406336, P25=3.8910916766663854, P26=1.112607860249405, P27=0.4817697213585717, P28=0.6647486612341246, P29=3.0052371058572245, P30=2.3055060391855537, P31=3.4504055963200857, P32=-0.1581598494038546, P33=2.124784266603632, P34=-3.7633772400081815, P35=-0.024332207724901578, P36=-1.1092758190557621, P37=1.5747593127361879, P38=-0.15794652788675534, P39=0.8210738322602493, P40=0.7223617162419338, P41=0.7123662615858964, P42=0.6854721945790684, P43=0.6916550139068881, P44=0.6610277251664443, P45=0.7206831144121026, P46=0.7183458456975418, P47=0.667770808371082, P48=0.6752335312952851, P49=0.7137676345883204, P50=0.7236209633902915, P51=0.7146586254594736, P52=0.00012675278890506055, P53=2.8837237408205337e-05, P54=3.8831799049720794e-05, P55=5.886382234791945e-05, P56=0.00012196526428958059, P57=0.00034503950736065707, P58=0.0005935129990441464, P59=0.0038174763460367966, P60=0.01070997756924712, P61=0.015976058171501634, P62=0.05119310649440138, P63=0.05254999790886875, P64=0.0182728741300603, P65=0.004871829771598075, P66=0.00344476506261175, P67=0.007595104493083586, P68=0.01219729320131775, P69=0.014082008508295397, P70=0.015042612738964822, P71=0.009762052167043202, P72=0.014087096242452059, P73=0.045547013629180184, P74=0.0449739596164845, P75=0.03051538923495385, P76=0.03646254980507251, P77=0.05471130376918998, P78=0.03485767161257662, P79=0.027405072072598485, P80=0.02116567811879335, P81=0.013832173630379417, P82=0.01715420665132642, P83=0.016210203846242766, P84=0.023262789582579553, P85=0.026235442107962796, P86=0.012580271275031725, P87=0.00825577199814872, P88=0.009313228817466003, P89=0.0079402722819931, P90=0.0025937072615230742, P91=0.0014918964614070614, P92=0.006656440818140874, P93=0.018090629887117387, P94=0.02808698283732127, P95=0.011287913264501263, P96=0.004437103717598647, P97=0.004378324198094902, P98=0.0047007268424176, P99=0.004010016237465615, P100=0.006448888368214907, P101=0.00467846409514154, P102=0.0013974533611901831, P103=0.0009247235213329926, P104=0.0011242463617530144, P105=0.0032466089861894263, P106=0.00887801982371272, P107=0.002753481938587959, P108=0.0005849700342640142, P109=0.0002973255469029949, P110=0.003478824578165804, P111=0.01163340204064379, P112=0.003107965272157363, P113=0.0008486001716143034, P114=0.00041797266455945067, P115=0.0005665341106089007, P116=0.000427924438841593, P117=0.0005989129574641349, P118=0.0007884055158818865, P119=0.0005515434437209553, P120=0.00038271441864700864, P121=0.000902981546477728, P122=0.0010677911753257626, P123=0.0003432983164680775, P124=0.00046501807949727263, P125=0.000579750258034422, P126=0.0013773937710155973, P127=0.0013867640168780254, P128=0.0006715126467448338, P129=0.0011655373861555248, P130=0.000531462315979662, P131=0.0005313377600868334, P132=0.0006037186043477998, P133=0.0004078582219519384, P134=0.000555358022839716, P135=0.00031975492272456885, P136=0.000433368759468561, P137=0.00015620351566564083, P138=0.00010844350785519483, P139=8.737481031362648e-05, P140=7.381695111110399e-05, P141=9.961179105311222e-05, P142=0.00011215516194876485, P143=0.00014403555633773197, P144=0.00014632344131814288, P145=8.531606349870283e-05, P146=9.772015782971732e-05, P147=9.413041756013735e-05, P148=8.017033558054541e-05, P149=8.854602049464608e-05, P150=0.00013161190512564315, P151=0.0002544446929033057, P152=0.00012503228937444586, P153=0.00011228430976619563, P154=0.00013855229801877457, P155=0.0001313070044910626, P156=0.00020122262377362935, P157=0.0002793495415824622, P158=0.0003717547627066941, P159=0.0003318880503808035, P160=0.0002197623639011144, P161=0.00019277819748083412, P162=0.0001019969032080292, P163=7.046636697993987e-05, P164=7.46531646466256e-05, P165=0.00014364719090011416, P166=0.00020823028766858568, P167=0.0001354643541953843, P168=5.6115806634000406e-05, P169=6.973712270180235e-05, P170=5.758677751378817e-05, P171=2.8403790692052054e-05, P172=6.384482512670497e-05, P173=0.00013776558121766678, P174=0.00011993890740936924, P175=0.00010269991037668991, P176=9.532242008056238e-05, P177=0.0001793918390716169, P178=0.00016132082447154616, P179=2.0933539081371334e-05),Row( P0=-427.8877782355205, P1=37.27414088797545, P2=-27.208600359551557, P3=-11.587171812219639, P4=-12.596492979557828, P5=-15.800796829922916, P6=-17.977519038806715, P7=-0.9233314919463226, P8=-8.089639641870148, P9=-6.50986718626029, P10=-7.339670605229283, P11=-8.741355298886605, P12=5.3581069643031505, P13=-1.3663672542332488, P14=10.988648534938905, P15=12.809620136716044, P16=7.0604186806144345, P17=7.190891423806283, P18=0.8602987172560588, P19=1.6065761949120587, P20=-1.097681349062521, P21=5.8674314921356485, P22=3.3653301629480006, P23=-0.9479492374206292, P24=-0.9810566309391425, P25=4.671164579882066, P26=0.8842257232651282, P27=3.511164345214763, P28=1.5767613079365903, P29=2.1010880939811725, P30=-0.03931374139450278, P31=-0.7836918089728424, P32=-0.943678363954471, P33=2.871358170329444, P34=1.8087281524460719, P35=-0.8939162946239393, P36=-2.0506141470821366, P37=2.020858831427813, P38=2.826939520310354, P39=1.5869334355815228, P40=0.6353179500841295, P41=0.5824789398418482, P42=0.5793755084494282, P43=0.6411938393121138, P44=0.6637021110909201, P45=0.5747546480411118, P46=0.5422236155477669, P47=0.5729850639021167, P48=0.606946671202099, P49=0.6421422118866048, P50=0.6512781974424892, P51=0.6477558062409783, P52=0.018788670624497707, P53=0.0013659705931501134, P54=0.0001025950514117068, P55=0.00011402361445403956, P56=0.00044011790625404594, P57=0.0008923618695618348, P58=0.0007766141854465179, P59=0.0017247719534786042, P60=0.005383527345592473, P61=0.015930959677408606, P62=0.058128658012350634, P63=0.09438337489834317, P64=0.08518648443844078, P65=0.15176029138400696, P66=0.24581585043262408, P67=0.4125872016918914, P68=0.53313997655882, P69=0.9053614859954685, P70=0.765307835963493, P71=0.35370245290206975, P72=0.2759474570164091, P73=0.3307589190274674, P74=0.15999518085133538, P75=0.04537140746924247, P76=0.05842238835064132, P77=0.4500367065302975, P78=1.9498816932129275, P79=4.637930681316052, P80=4.630158090270174, P81=1.4358115455607339, P82=0.26917354143929123, P83=0.29893450756623086, P84=0.44470851041181464, P85=0.5097446675843943, P86=0.4588173248863899, P87=0.5579642281648761, P88=0.8894922954750364, P89=0.7031050042577925, P90=0.8626939018294788, P91=0.8086265840443219, P92=0.44266070584805356, P93=0.19700104891947054, P94=0.05612012400613393, P95=0.30190619061057344, P96=0.2462190774310162, P97=0.4219228454298718, P98=0.4697351864826245, P99=0.3925438918364893, P100=0.10713382637283854, P101=0.09542936162754624, P102=0.11155552711350256, P103=0.19212903761912584, P104=0.12096786358462142, P105=0.07480385238160045, P106=0.1073637606509594, P107=0.05510604724813619, P108=0.02170509656104275, P109=0.021981711650839654, P110=0.12221654203514262, P111=0.284212771625124, P112=0.22376369061370482, P113=0.2450074642588319, P114=0.02532334557644074, P115=0.012824057139869732, P116=0.027234240747378558, P117=0.03296873700607723, P118=0.03071608834238123, P119=0.02500440445128089, P120=0.032890249995917775, P121=0.04891257945865774, P122=0.034435496564493476, P123=0.0315690136222072, P124=0.017187723505438065, P125=0.03246315037828786, P126=0.019227823783568368, P127=0.00903611132763556, P128=0.016076936193925823, P129=0.014835522979381973, P130=0.007202445721368941, P131=0.0037196111951163283, P132=0.005897426379354169, P133=0.00916193303224297, P134=0.015080100566382414, P135=0.011460124906150145, P136=0.008464366553792122, P137=0.0039056140861485184, P138=0.0015840940451580397, P139=0.0019387235889188551, P140=0.0021499689185932023, P141=0.0005781103089271938, P142=0.0005001301999205482, P143=0.0007480780820727598, P144=0.0008770861421304192, P145=0.0012045086989383436, P146=0.00177804993267939, P147=0.00039703511220590647, P148=0.0004224945345906818, P149=0.0003342951403077141, P150=0.0004846990090921124, P151=0.00033350545461297845, P152=0.0005515829741809071, P153=0.0009130211137807783, P154=0.0011230572158018094, P155=0.0020136465811465, P156=0.0030807358529895803, P157=0.002127667266984029, P158=0.0023248725150347197, P159=0.0026463469617191964, P160=0.0023486297589668708, P161=0.0024080744623883447, P162=0.0025477522893688504, P163=0.0017035423128875789, P164=0.0016092879321015143, P165=0.002916959952346158, P166=0.0024164546921211926, P167=0.0015532674981912386, P168=0.0012987368361689763, P169=0.002408440011651623, P170=0.002934129596961255, P171=0.0029674496549972514, P172=0.0019825070987155064, P173=0.0013896992648782353, P174=0.0009852246398196962, P175=0.0012748766897938777, P176=0.0037879854460176644, P177=0.002691411203650046, P178=0.0012995545071116266, P179=0.00011925787161926288),Row( P0=-653.9323750072496, P1=69.2690910404549, P2=-15.130772338307244, P3=-0.17358692062301573, P4=-10.632106848244126, P5=-18.14474002488298, P6=-5.264487079455615, P7=6.269127758496436, P8=-15.893578764249616, P9=-3.3406169751616694, P10=-5.595467973274292, P11=-10.263195613911636, P12=0.1281945752016594, P13=-12.996287605023808, P14=-4.596387252703981, P15=-5.007758058643011, P16=-4.765406343546023, P17=-2.9492685031553316, P18=-6.725715348475604, P19=-4.340716080934578, P20=-6.461558669504494, P21=-4.836944728488046, P22=-6.2295461149594376, P23=-5.77429315159705, P24=-6.305818889795431, P25=-4.224614766993046, P26=-3.8978552780863067, P27=-3.941303118735544, P28=-1.5610742840194962, P29=-1.998197610248871, P30=-4.351975723531677, P31=-1.372211277980098, P32=-1.196030997356387, P33=0.6618417778377662, P34=1.5573360142777168, P35=2.312080707267556, P36=2.9530374220721307, P37=5.9381006634026505, P38=1.6743655336299077, P39=3.6651877778882684, P40=0.6682277148919353, P41=0.6682577545214701, P42=0.6399379974589248, P43=0.6104595428906862, P44=0.6254517926686476, P45=0.6382936142832693, P46=0.6292071275948725, P47=0.643699129811585, P48=0.7026033083186736, P49=0.7702986281589693, P50=0.7240497954942235, P51=0.6722938039299363, P52=5.406831889271242e-07, P53=1.8644785352882367e-06, P54=3.7664102516335595e-06, P55=1.3134905366884095e-05, P56=0.0008188417313092363, P57=0.00930707290816534, P58=0.016254861229672516, P59=0.016225478504214998, P60=0.015967773090137624, P61=0.013721765883410847, P62=0.0024006878492924314, P63=0.0007982786113875963, P64=0.003122975674779676, P65=0.013191232786877534, P66=0.019785506854282302, P67=0.01950414993749863, P68=0.00717288174503304, P69=0.006299349607254277, P70=0.01771247012724067, P71=0.008927537697894108, P72=0.0025433178897333714, P73=0.0034042713024326535, P74=0.0033023122494051788, P75=0.004520352191489737, P76=0.003426449211911728, P77=0.003278679055303209, P78=0.006675354080984142, P79=0.014534554716323913, P80=0.015393577716107939, P81=0.008428169362804138, P82=0.006280586528395163, P83=0.009034023830654768, P84=0.009172003444564846, P85=0.006068137596286719, P86=0.006061401131619758, P87=0.0052365847091306456, P88=0.014413949721145243, P89=0.013910377195621198, P90=0.00618354368747029, P91=0.004641467935891273, P92=0.003462580034453317, P93=0.002885823700641843, P94=0.00429097484853448, P95=0.009181533779563515, P96=0.0059357336971193305, P97=0.003079852696774146, P98=0.0026421325439506607, P99=0.003951467073844089, P100=0.0037081957474223294, P101=0.003937631204871162, P102=0.0014896079091395657, P103=0.0008980784334838035, P104=0.00047996352965493306, P105=0.0009346819358370561, P106=0.001219104432976438, P107=0.0005323937581976462, P108=0.0003485559247161343, P109=0.00039679484160259654, P110=0.00025487678598440697, P111=0.000466741173277686, P112=0.0008078218047649519, P113=0.001246408159966885, P114=0.00030231291946028883, P115=0.00031706659628358757, P116=0.0004559569715092494, P117=0.0003023739286300581, P118=0.00034051397831417145, P119=0.0006196000059403593, P120=0.0001708284553156669, P121=0.00017813507962626356, P122=0.00023246408215161658, P123=0.0002382630948178203, P124=0.0003160893680843038, P125=0.00034176531709888166, P126=0.0003302945497907176, P127=0.0002906626962579525, P128=0.00022733097538937686, P129=0.00012672580450791203, P130=0.00010611438109672352, P131=0.00017053127974599137, P132=0.0002581419688845138, P133=0.000264198654089026, P134=0.00041408840951076906, P135=0.0003152034783095065, P136=0.00019189218132018663, P137=0.00012316769566774716, P138=5.977852573708811e-05, P139=8.652483319518398e-05, P140=8.213649204762813e-05, P141=5.6445520090783434e-05, P142=3.6687354166139805e-05, P143=2.8053050430760032e-05, P144=7.402164317923499e-06, P145=2.4290334045312057e-06, P146=1.6968215459644988e-06, P147=2.3576208903260614e-06, P148=2.644035926651714e-06, P149=2.6674022309416786e-06, P150=4.467994650014662e-06, P151=6.630748160674581e-06, P152=1.4009520984114762e-05, P153=9.89696133637782e-06, P154=1.2205252902251071e-05, P155=1.9265514033453758e-05, P156=2.7472168372430053e-05, P157=5.023833536490683e-05, P158=7.552596454061839e-05, P159=0.00011176007711411422, P160=0.00019508085877507266, P161=0.000268035035802115, P162=0.00026269640626228193, P163=0.0002756811396605949, P164=0.000248052845814872, P165=0.00019549029971169778, P166=0.00012766476029173177, P167=8.658304095764679e-05, P168=4.834962553916856e-05, P169=8.56174567716498e-05, P170=7.635025722949455e-05, P171=4.156585121367154e-05, P172=3.42774968880172e-05, P173=2.1144123871144065e-05, P174=1.4031206933136896e-05, P175=1.6753731318617938e-05, P176=2.5178009630890237e-05, P177=2.701016435900215e-05, P178=1.2602045439337466e-05, P179=2.17301166470367e-06),Row( P0=-549.1245216158003, P1=53.02333195513309, P2=-24.3833686292426, P3=0.1175326097024951, P4=-13.125209301815778, P5=-16.001361945505376, P6=-6.2106282268725, P7=3.9184852674082724, P8=-3.6523734992404333, P9=-6.322781157592939, P10=-4.090708816989412, P11=-6.494948173750802, P12=1.3090558708389972, P13=-8.760927599850435, P14=-3.1650936811067467, P15=-2.016340729900147, P16=-3.04237813715041, P17=-0.06999273714020471, P18=-9.033706709653464, P19=-2.7616777904477576, P20=-4.912614956515508, P21=-1.1927723838434097, P22=6.736874002344359, P23=4.650667477251455, P24=7.925865094812247, P25=10.606772968873067, P26=7.131309069695168, P27=7.1361655090271885, P28=4.042595505610696, P29=3.9217330265131, P30=2.2379058721157983, P31=4.435837254625908, P32=2.492018931863525, P33=5.7724857958071105, P34=1.952196364074347, P35=-0.07450616783530759, P36=-0.8424871521598107, P37=0.7864423516858511, P38=0.742904995186753, P39=2.2632443438074876, P40=0.6379532626150448, P41=0.6006725793331852, P42=0.5969817730683606, P43=0.6404089867487652, P44=0.6888294296823999, P45=0.676608185671147, P46=0.6086456218946713, P47=0.6036637424689087, P48=0.6201943887604251, P49=0.6046539554381521, P50=0.6142008507185249, P51=0.6320356531962783, P52=0.0011107674191290974, P53=7.727103256323138e-05, P54=2.2686654325224217e-05, P55=2.5568502946764256e-05, P56=4.198617746806628e-05, P57=0.00011080575851305242, P58=0.0023235475269310097, P59=0.014155942835303267, P60=0.046092784012303836, P61=0.14133107203831508, P62=0.1993266808968915, P63=0.09364907178649452, P64=0.01314058520541244, P65=0.001974124294044122, P66=0.002273650104277276, P67=0.006699867428304867, P68=0.016131262186142727, P69=0.030312616893033766, P70=0.03393174939073925, P71=0.058897978059806205, P72=0.14273013956802638, P73=0.1264140414086195, P74=0.09698756406581874, P75=0.11737539843383876, P76=0.04501581769936072, P77=0.01636850804560731, P78=0.017684075161190922, P79=0.01319166366420529, P80=0.014395074748183662, P81=0.037728440787630754, P82=0.07767209438061935, P83=0.07573111220133075, P84=0.057785714786785486, P85=0.04273520822052637, P86=0.02902764408010317, P87=0.022332257988558354, P88=0.02795063581340888, P89=0.023566516175260892, P90=0.049637624301600014, P91=0.09586053775057492, P92=0.05368886372605634, P93=0.02984441964720985, P94=0.05598230308594172, P95=0.020502829980274404, P96=0.009539924541560416, P97=0.12585756532148884, P98=0.24948325409238525, P99=0.0726251875226453, P100=0.024849196341786527, P101=0.029295555274704618, P102=0.007761264793790739, P103=0.02142199170746221, P104=0.018222195269870935, P105=0.004015050651806416, P106=0.011556365386739617, P107=0.011249244199411625, P108=0.01452195680929416, P109=0.005049549618519956, P110=0.005886633576602574, P111=0.051327121552788145, P112=0.05251296700158961, P113=0.022512254942715672, P114=0.00108546076330841, P115=0.002676340527515617, P116=0.011555785437497345, P117=0.014996362799361795, P118=0.0013526339756059626, P119=0.003764328845351674, P120=0.008700026013403191, P121=0.001722350417120251, P122=0.0038740807392860246, P123=0.004021533307403358, P124=0.0015912065935963602, P125=0.0028612242063842277, P126=0.0030997024845519135, P127=0.0030590695603088668, P128=0.001544920668415502, P129=0.001152305322891378, P130=0.0015224756828579212, P131=0.000830084774051288, P132=0.0013317926078307623, P133=0.0009292831671201087, P134=0.001625688187965893, P135=0.0014324158365272334, P136=0.0036183147981952795, P137=0.0007489907673131433, P138=0.0005767538544123103, P139=0.00039903773976463766, P140=0.0003289828267068794, P141=0.0003552450947206993, P142=0.0005034817828382634, P143=0.00047884857372584454, P144=0.000445081525760079, P145=0.0002954940762446901, P146=0.0002781034882997539, P147=0.00015452980339644864, P148=7.335169603409616e-05, P149=5.513319550925953e-05, P150=8.814046159012659e-05, P151=0.0001499260486782616, P152=0.0002231768289429889, P153=0.00027396178666664554, P154=0.00021328622481914735, P155=0.0002405579106860971, P156=0.0003545908087338068, P157=0.0005348554954107274, P158=0.0004867185699957646, P159=0.0005150069062195466, P160=0.0005219337542334695, P161=0.0004567708409141911, P162=0.00025536494325407804, P163=0.00018551699289264963, P164=9.338849848672485e-05, P165=0.0001530350299392019, P166=0.00019628093161125983, P167=0.00020617377642885264, P168=0.00011575984607658268, P169=0.00013911741958425508, P170=6.515009759129035e-05, P171=7.428346046024444e-05, P172=0.0001549040688831183, P173=0.00014572025098261903, P174=9.974685865238054e-05, P175=5.6304133609835755e-05, P176=5.710782153014203e-05, P177=0.00011793255506762103, P178=8.039658469645248e-05, P179=1.049264441662719e-05),Row( P0=-653.0028985262633, P1=65.31547050145511, P2=-5.479347504753122, P3=0.008690831726036012, P4=-10.791042490862008, P5=-11.753340378351737, P6=-6.038037650808492, P7=2.222515377936366, P8=-8.835122771477721, P9=-6.771704153831764, P10=-5.185880880137807, P11=-8.915456205496533, P12=-1.704344871784193, P13=-10.043412698501143, P14=-4.790058452687269, P15=-3.5725122442255195, P16=-4.434019873164685, P17=-0.6049672659725086, P18=-7.133771282316866, P19=-3.2027537784006865, P20=-7.273350329643085, P21=-5.826799254609374, P22=-3.8577482429460312, P23=-5.446140890182025, P24=-2.993344853322892, P25=-2.428157891294272, P26=-2.1821411875755343, P27=-0.6394150925399951, P28=-0.34023463512631263, P29=0.9196292171185624, P30=-0.8824284026762438, P31=3.4015578201587053, P32=0.3741256695937572, P33=2.2911727977248626, P34=-0.9159147947956621, P35=-1.2874195580129923, P36=-2.7945499783959113, P37=-0.7055398951490526, P38=0.19449743032484196, P39=1.1572363523250233, P40=0.7122873649147742, P41=0.6684269159918172, P42=0.6459540556889481, P43=0.656477368116279, P44=0.6547595921810899, P45=0.635439340439316, P46=0.6372448154869186, P47=0.6629562189472509, P48=0.650718995358182, P49=0.6445204302991192, P50=0.6407958813301803, P51=0.6647081280644185, P52=2.034916999424597e-05, P53=4.347387922543659e-06, P54=1.1923740717431641e-05, P55=3.2450588853804364e-05, P56=0.0011330604658742072, P57=0.008205887283254994, P58=0.006623817656310788, P59=0.011071227536160806, P60=0.012781843595161105, P61=0.013624349562579505, P62=0.008119267836057015, P63=0.003288617627439741, P64=0.003268946088215746, P65=0.0032575524315180526, P66=0.004900352342852448, P67=0.01467683224370088, P68=0.028432420491715082, P69=0.01608027712287084, P70=0.009950601121830487, P71=0.013126541063780157, P72=0.00719407376638784, P73=0.002452963109543065, P74=0.001141287400383063, P75=0.00231417744020598, P76=0.004174017696979286, P77=0.01028273168812629, P78=0.00946694163192437, P79=0.007368386941833569, P80=0.020467581932500426, P81=0.030422001674232924, P82=0.01460980180133102, P83=0.006966541870665057, P84=0.0064670630018410355, P85=0.004656147286901667, P86=0.002292634689539082, P87=0.0014416332676379466, P88=0.0017387631647326927, P89=0.0024334945187303807, P90=0.0067567975496257236, P91=0.0066175116382750495, P92=0.004889082610725572, P93=0.0016428118327623247, P94=0.0015643668533577002, P95=0.005566351117506811, P96=0.008672066112085017, P97=0.007089223611359782, P98=0.003352304380679896, P99=0.0013862667296807023, P100=0.007532570032674302, P101=0.006461524618652752, P102=0.0017930387357311902, P103=0.000492324465070267, P104=0.0012883297754250787, P105=0.001871290279522599, P106=0.0020934542319139683, P107=0.0006641733220703314, P108=0.0007501135380451753, P109=0.001034289866156798, P110=0.0008287671021049351, P111=0.0029020390215628486, P112=0.0017751031962655547, P113=0.0006052723312651179, P114=0.0005957545578210009, P115=0.00034699865334699066, P116=0.00022937953823265036, P117=0.0014820887284536716, P118=0.0003175116456540613, P119=0.00022555897412932928, P120=0.00036102721808667495, P121=0.00014955012802791068, P122=0.0005287953605366928, P123=0.00025930534567651333, P124=0.00032102375852268973, P125=0.00023258904383354091, P126=0.00022790936150462367, P127=0.00025833233917546246, P128=0.0002412040806116674, P129=0.00015711442000459163, P130=0.00017534271127726196, P131=0.0001528966811977717, P132=0.00018311852302218098, P133=0.0002358008524392846, P134=0.00032419516634140993, P135=0.00016717946564156715, P136=0.00015913944111035994, P137=6.959045022610765e-05, P138=3.894419282966523e-05, P139=3.836720711593879e-05, P140=3.520129116924848e-05, P141=2.7891111029630983e-05, P142=3.8198710619379464e-05, P143=3.975430600768045e-05, P144=5.538767115911138e-05, P145=2.5720590039179575e-05, P146=2.3662517400880578e-05, P147=1.1436680608270027e-05, P148=1.6020921654746814e-05, P149=1.5069445701026944e-05, P150=2.2958172806220064e-05, P151=1.7710749522460124e-05, P152=2.1784693729031597e-05, P153=2.3921029337289407e-05, P154=2.0890145402714745e-05, P155=2.7440620310899192e-05, P156=6.355545669762835e-05, P157=6.873327677061235e-05, P158=9.590458348536694e-05, P159=0.00011915342568954697, P160=9.440837418148495e-05, P161=4.026315335988045e-05, P162=2.8801514474084283e-05, P163=2.598497632461026e-05, P164=1.6367091226581886e-05, P165=2.8789368130152916e-05, P166=2.8849273815835133e-05, P167=2.2151018280993068e-05, P168=1.90693121816108e-05, P169=1.5130066910218342e-05, P170=1.4856374957666952e-05, P171=1.7943746184799307e-05, P172=2.1907859916333444e-05, P173=1.633686458986063e-05, P174=2.286304742346298e-05, P175=1.4476646488737711e-05, P176=7.926343317029977e-06, P177=6.717997421885765e-06, P178=4.29525717282964e-06, P179=1.2928663445652344e-06),Row( P0=-544.4524277582447, P1=50.93426031673289, P2=-30.850911290233782, P3=-9.47017730792229, P4=-16.139074741657165, P5=-15.877676758290054, P6=-17.74276041243598, P7=-2.9577211392830343, P8=-8.131172562061762, P9=-9.114188076306997, P10=-9.679143635578066, P11=-11.20338449439016, P12=-0.12034152110119742, P13=-8.970453037582438, P14=1.2753977651645716, P15=4.589128380159413, P16=4.4207740342196455, P17=10.569753641263766, P18=5.570116763822091, P19=5.732043215829547, P20=-1.3270442179071202, P21=-0.25167715448304684, P22=-0.32294003514475816, P23=-3.1471314406750412, P24=-1.2559976519581066, P25=3.0241272655823517, P26=-0.21382885540749777, P27=1.4826183756542979, P28=-2.3353661721374057, P29=0.6933341125067429, P30=-0.3251748206727183, P31=1.3752618113080792, P32=-2.2546587029456404, P33=-0.5172794954234746, P34=0.40271590015727954, P35=-1.3738981433164117, P36=0.2589384840072649, P37=1.0320777038781395, P38=-0.31625683315847414, P39=1.1397679089421946, P40=0.6549129909335957, P41=0.5647426222217007, P42=0.5746033161031335, P43=0.6210619309748261, P44=0.6622211224312304, P45=0.655816681361748, P46=0.6115949141985217, P47=0.5982836765539243, P48=0.5943073993069475, P49=0.6775983212684864, P50=0.7075959679131614, P51=0.6722486344884661, P52=0.00015023238753623732, P53=4.792749629302578e-05, P54=2.0300419633952555e-05, P55=3.2695824664611804e-05, P56=0.00015114723181211648, P57=0.0005289327675992332, P58=0.0007628341788167113, P59=0.0008044923632357696, P60=0.0007693936283697153, P61=0.0032654169588452295, P62=0.02062292939648701, P63=0.0457170321170145, P64=0.0526307254183665, P65=0.16547455786248297, P66=0.24754271103116848, P67=0.23012428240975277, P68=0.13531506257847875, P69=0.12067467651541079, P70=0.23307657448709498, P71=0.14026910470382334, P72=0.013123439503182935, P73=0.008511955003743354, P74=0.005092464325015001, P75=0.004088293179026507, P76=0.007497057674182161, P77=0.03495427784051803, P78=0.08776581824575307, P79=0.15672290059597865, P80=0.23419431496471063, P81=0.20101781571217664, P82=0.15466298514145824, P83=0.2078672943146354, P84=0.30924754775409496, P85=0.2994623269492101, P86=0.16958980248312402, P87=0.14493798817879042, P88=0.16631699743638703, P89=0.049038341978627324, P90=0.048956790744629074, P91=0.16612495030806065, P92=0.3589994868612816, P93=0.15771507960666536, P94=0.15394552569978415, P95=0.22121059534876822, P96=0.16107778682070104, P97=0.04015365697394737, P98=0.0192400668993635, P99=0.08239273075033038, P100=0.11878489539931139, P101=0.04983493241026778, P102=0.004451997788458145, P103=0.004955916437638011, P104=0.009498191529772743, P105=0.012434571226548407, P106=0.023742589018882557, P107=0.018159939683070166, P108=0.006746369726487929, P109=0.008036634663664987, P110=0.019728054572628363, P111=0.05312054523496744, P112=0.0632426466801716, P113=0.02797737832319407, P114=0.0045650795682558015, P115=0.0022188215130691555, P116=0.004081548689126296, P117=0.007862835845895911, P118=0.016703070479613065, P119=0.01737897693918245, P120=0.0054816517058047885, P121=0.007414024554724426, P122=0.005326147982990604, P123=0.004181952463190639, P124=0.001771587209767353, P125=0.0018658893257326192, P126=0.0040181324473177755, P127=0.0032379689648850437, P128=0.0015500711099792825, P129=0.0011740572088503522, P130=0.0007585140646328437, P131=0.0009798546887519464, P132=0.001138739084248029, P133=0.001871015056684555, P134=0.0018229086031662715, P135=0.002057927483930465, P136=0.0011585116833388468, P137=0.00044474613125179143, P138=0.00032563396550143214, P139=0.00033079142070824156, P140=0.0003626582913215108, P141=0.00029802039746552265, P142=0.0003523138497380608, P143=0.0003353567134824083, P144=0.00046633116617395676, P145=0.0003025370547133229, P146=0.00016685889501865758, P147=0.00013311883932894925, P148=0.00014842574154142496, P149=0.00016418801915460301, P150=0.00013352097007791967, P151=0.00010426169939536284, P152=0.0001116791124473624, P153=0.0001410844703981212, P154=0.00019529079948515918, P155=0.00026442090472524244, P156=0.00048631663841527544, P157=0.0008383119624663962, P158=0.0007012666287205682, P159=0.0004961810809018494, P160=0.0003543132436855306, P161=0.0002939725742400783, P162=0.0003321556194586459, P163=0.00029359434604051606, P164=0.00017706377815639095, P165=0.0002560318788111059, P166=0.0003868313239324501, P167=0.0003181334861937763, P168=0.00016238324763306628, P169=6.244010776529218e-05, P170=5.845289222439995e-05, P171=7.339201514841851e-05, P172=0.00015362263336326482, P173=0.00018034286012981316, P174=9.004458268128039e-05, P175=5.535899287594251e-05, P176=8.586687936373876e-05, P177=0.00014666330374573912, P178=7.813730831590312e-05, P179=7.73132253500295e-06),Row( P0=-561.1900471872408, P1=55.347578898997625, P2=-23.69004336896594, P3=-3.222354114109174, P4=-14.85569855331974, P5=-18.206991428094526, P6=-12.876718883426978, P7=0.5675935899614974, P8=-5.932215911256453, P9=-7.930592385384668, P10=-9.798699535409112, P11=-10.187126429818504, P12=-1.6621942447575464, P13=-12.067534698423824, P14=-4.740267730155404, P15=-1.4656890210804665, P16=-2.1972737803558084, P17=2.642660667458073, P18=-1.0352489287674411, P19=2.7665560521251122, P20=0.004625620067480894, P21=0.17060295840776055, P22=-0.738997476842488, P23=-0.17185105403056544, P24=2.194002894760785, P25=6.245345738192664, P26=5.7279836824076, P27=4.240804872626313, P28=2.177705988472171, P29=-0.9163969850754977, P30=-2.8873436393614806, P31=1.3000995221427352, P32=-0.6562708802127463, P33=2.081023621538378, P34=-1.6736099359244336, P35=-1.9250020454597365, P36=0.1962664984763785, P37=2.178379057369981, P38=2.7333992119338397, P39=0.35582743841089337, P40=0.6324243660172567, P41=0.613592429292539, P42=0.5694733448633333, P43=0.6096679412529591, P44=0.7143012121360788, P45=0.7572196875256724, P46=0.6824840697274623, P47=0.640731874395248, P48=0.6324023930766358, P49=0.682275564840221, P50=0.7385941929139652, P51=0.6778629835539932, P52=0.003070518680868178, P53=0.00013793218263547638, P54=8.310001210433272e-06, P55=1.9753162886576367e-05, P56=4.416320343919257e-05, P57=0.0002978616643847757, P58=0.0012628522841656017, P59=0.0017948799040842132, P60=0.013494016513164129, P61=0.059133710859736836, P62=0.06905119389300479, P63=0.03430392087497328, P64=0.017864405655394214, P65=0.05644181671582737, P66=0.06530195726963761, P67=0.033895389824018504, P68=0.014320175992119565, P69=0.019861789355125396, P70=0.045294279933618314, P71=0.04758933046604929, P72=0.035425584072756616, P73=0.03302119512862999, P74=0.03617680887508683, P75=0.030099291740887508, P76=0.014242187705023234, P77=0.026239269322141604, P78=0.12249238067546608, P79=0.3846304456835675, P80=0.40222795662522653, P81=0.2235742671678481, P82=0.09888311982105283, P83=0.0551746923989887, P84=0.08186443087076063, P85=0.08586681435406439, P86=0.04120191378736864, P87=0.03054911333941667, P88=0.032721556999157055, P89=0.04007341587758007, P90=0.09685116901652922, P91=0.20985509367922955, P92=0.24205132532831278, P93=0.07820807292058453, P94=0.01012654918875854, P95=0.002552018123049847, P96=0.0012561695845416032, P97=0.01319479359464319, P98=0.062283246377077967, P99=0.046222789794520214, P100=0.025506977288266886, P101=0.017413107569611862, P102=0.00700542925612337, P103=0.0032055838820921716, P104=0.004769438327834684, P105=0.002744512562378607, P106=0.0038956453912197303, P107=0.009543723711606136, P108=0.003813191157255119, P109=0.002496923944268698, P110=0.0007787456107840303, P111=0.0017953069685498605, P112=0.01296292498647272, P113=0.017543665081837342, P114=0.0022909264089226544, P115=0.001862825205557835, P116=0.0032642726447478436, P117=0.007106235417815538, P118=0.014316046646281966, P119=0.01738539976659108, P120=0.004050174697301676, P121=0.005460188849023312, P122=0.005123646898066436, P123=0.0021481469251187543, P124=0.0018796189676903198, P125=0.0014670125174662658, P126=0.005092837568138736, P127=0.007855984476501307, P128=0.0027078137116433357, P129=0.0017472130018463439, P130=0.0008996052143637545, P131=0.001465650411694873, P132=0.000942705441943189, P133=0.0017746875262511985, P134=0.0021701561818417, P135=0.0011422292517522556, P136=0.0011467160589644825, P137=0.0006281490742244022, P138=0.0004364588498992032, P139=0.0005013626758896786, P140=0.0005341549979061094, P141=0.000526411803982265, P142=0.0005067970331271031, P143=0.0002549116266625246, P144=0.000396567335852923, P145=0.00029596994455828276, P146=0.00019854530552008168, P147=8.545480686515891e-05, P148=6.34195870921032e-05, P149=5.072694538188572e-05, P150=6.487352521136389e-05, P151=7.070279564563995e-05, P152=9.146252205780092e-05, P153=0.00010804792898363965, P154=8.482843218848186e-05, P155=8.677663997602165e-05, P156=0.00015167454879703226, P157=0.00021461526047750776, P158=0.0002127586608114239, P159=0.0002961864745910988, P160=0.00031329069281189014, P161=0.0002336319263308555, P162=0.0002383001542351521, P163=0.00018951669338953007, P164=0.00016370150995602628, P165=0.0003672765492308773, P166=0.0004968873366874002, P167=0.0004683658818621122, P168=0.00020291494099202703, P169=0.00022603309712183414, P170=0.0001889569692976319, P171=0.00014010505094041572, P172=0.00013580774867922045, P173=8.655933260263228e-05, P174=6.34371759910224e-05, P175=7.045386590080223e-05, P176=5.3603271495678764e-05, P177=6.925742697801842e-05, P178=4.8898400401111096e-05, P179=7.027136708665215e-06),Row( P0=-390.9260268093175, P1=64.9751483737148, P2=-17.226908327593208, P3=11.106617554514559, P4=-8.188384633405262, P5=-10.534165026614328, P6=-6.45258102980119, P7=3.062519576027029, P8=-0.02592679137696574, P9=2.2309513343526253, P10=-3.3051335310375762, P11=-4.736002901354348, P12=7.4074921877705755, P13=-7.167327649906269, P14=4.163274794407967, P15=13.37753225654726, P16=6.928575379315575, P17=13.47626745328888, P18=8.453147191701841, P19=13.789830516856016, P20=5.386885177530931, P21=5.7318773853316, P22=8.776606081329989, P23=4.840043034958369, P24=3.7016037653893905, P25=7.408706341325879, P26=0.46106796371859743, P27=2.6981999375386123, P28=4.063358522659261, P29=4.223789815832548, P30=2.66196270934152, P31=4.540055076589419, P32=2.151110240181114, P33=1.4826125899574822, P34=-0.6987747864858422, P35=2.63799097833707, P36=-0.11125681217194965, P37=2.8148956624635826, P38=1.2650249348040261, P39=4.348390422981827, P40=0.633285036365993, P41=0.6025833634255694, P42=0.5637809258299301, P43=0.6177222867717183, P44=0.6500808324879033, P45=0.7123171309634936, P46=0.7449587346692351, P47=0.7343078257448111, P48=0.6994312859355541, P49=0.7156001538860282, P50=0.7233051614017291, P51=0.6888959039305724, P52=1.943572059467099, P53=0.47175094907975657, P54=0.03438293673300487, P55=0.004286089406438352, P56=0.002564965883347864, P57=0.003967642770940639, P58=0.0022786638440681, P59=0.0036489155152795365, P60=0.012448055084890493, P61=0.03258504943534823, P62=0.13016042115504922, P63=0.2902134359324438, P64=0.40220695360564435, P65=0.98047334087819, P66=1.551349372166807, P67=1.2972441876322367, P68=1.4892279899238676, P69=1.225347522688431, P70=0.5118410607337386, P71=0.19439835983320025, P72=0.09123403463949012, P73=0.09977120516978592, P74=0.14049115864320327, P75=0.24322560097549667, P76=0.40847171759927464, P77=0.8067507366616233, P78=0.8604827481759303, P79=0.9561940268462648, P80=0.9203777693357116, P81=0.6921634597939417, P82=0.7072075688202762, P83=0.5179723812874194, P84=0.66920566709488, P85=1.7618826625999182, P86=1.1118579250313179, P87=0.33555766256902403, P88=0.3210129993214141, P89=0.18313899906624168, P90=0.11669807505930564, P91=0.11187201322830066, P92=0.2325762965611202, P93=0.2517470704760836, P94=0.6359655845590867, P95=0.40486425656198555, P96=0.18892280164906064, P97=0.22367118694361027, P98=0.3711962433429499, P99=0.12000767419952975, P100=0.05075496761893746, P101=0.06962030118171292, P102=0.1178778335856798, P103=0.25359541317369033, P104=0.1250171156296224, P105=0.07416987820951788, P106=0.13425002248970644, P107=0.15477759451481327, P108=0.091807938388595, P109=0.07395185470377844, P110=0.2607034999921231, P111=0.2000400326118316, P112=0.05039791992713081, P113=0.1772242362753693, P114=0.1550309424769598, P115=0.08245085535708861, P116=0.02428475464552268, P117=0.03328203638300258, P118=0.055902842962120815, P119=0.06514203904467268, P120=0.06762170009402406, P121=0.07905857425292782, P122=0.03619011884604627, P123=0.04907926068414211, P124=0.14949753442777275, P125=0.07075404704092425, P126=0.05754846875330567, P127=0.048625719535984574, P128=0.0742804906908636, P129=0.01943945743504574, P130=0.023744108918021672, P131=0.02046062123241706, P132=0.02130752444849202, P133=0.019326237753821997, P134=0.018511044888383326, P135=0.036097621287567605, P136=0.02493969428941748, P137=0.014032198399994663, P138=0.01689651231544363, P139=0.008263369335394306, P140=0.014042085971909818, P141=0.01784163599842833, P142=0.009150272829968014, P143=0.009699933618443516, P144=0.00601953742966494, P145=0.003787309268219124, P146=0.0026498105067502948, P147=0.0017394745014382275, P148=0.0015829658835260548, P149=0.0020928675680837758, P150=0.0025132721570650008, P151=0.0022423244764969022, P152=0.0015841477492727837, P153=0.002038573071913507, P154=0.002262244697872971, P155=0.004089729873499921, P156=0.004005706469690414, P157=0.004664420595055851, P158=0.004609497028304224, P159=0.0036463677620049138, P160=0.003487556324094726, P161=0.002989183333871687, P162=0.0023681982637368164, P163=0.0018383594246321974, P164=0.0021077383195210225, P165=0.0015616048710458935, P166=0.0012478676907980767, P167=0.0007842041180103255, P168=0.0016249823194326282, P169=0.003179756022130092, P170=0.004218213840543268, P171=0.005874592947565133, P172=0.0033959265725756562, P173=0.0031071166053949525, P174=0.0019909710542567183, P175=0.0018353662083947292, P176=0.004935395770800375, P177=0.003590475943085969, P178=0.0015279348124216088, P179=0.00015338200295446195),Row( P0=-572.120563656382, P1=68.549872126625, P2=-34.758316660984235, P3=-5.420047106440263, P4=-16.12934002527536, P5=-12.589026922629888, P6=-5.24497411537064, P7=7.490941924694237, P8=-9.4026955997652, P9=-5.6822859513837125, P10=-6.396821376208168, P11=-12.94875526328139, P12=5.805650201243517, P13=-13.791791436472193, P14=-4.129358533492936, P15=-0.41352978530413786, P16=-4.708893945963892, P17=1.8501138156614931, P18=-8.899093296055533, P19=-3.916019286236647, P20=-5.242197610289446, P21=-4.88278111579143, P22=-1.8381083410088037, P23=-5.611233145231973, P24=-5.347357388042546, P25=0.6150232236133987, P26=-1.6986467037595439, P27=0.8193013785617311, P28=2.6132068482344804, P29=3.255047561819432, P30=1.8404208800202946, P31=2.020356040545232, P32=1.54581538157274, P33=4.168811948780052, P34=2.5268428519730577, P35=2.5034265393855053, P36=1.8467205468224004, P37=3.0566855511800015, P38=0.5391261870855437, P39=1.5290306604191333, P40=0.616153176099034, P41=0.5951340549917833, P42=0.6484612681600915, P43=0.6539438601046259, P44=0.6580259570185801, P45=0.6728618112478135, P46=0.647426469582507, P47=0.6406031944026975, P48=0.6471096760391848, P49=0.695007939548859, P50=0.6396357510779653, P51=0.5820035108095613, P52=0.0014938638125939771, P53=8.589218285454333e-05, P54=4.5393091546546055e-06, P55=6.991751069938284e-06, P56=0.00039279396500241587, P57=0.003542942311555603, P58=0.007511653756664873, P59=0.017642718030365545, P60=0.03342666121748581, P61=0.023766151925873107, P62=0.0046197730748186745, P63=0.0015527525078107378, P64=0.0014633438461116805, P65=0.005020398009097616, P66=0.012461598596323594, P67=0.020284943236647573, P68=0.027119614814675768, P69=0.06070650324825191, P70=0.06975203693065601, P71=0.036075637266457385, P72=0.001954605265693264, P73=0.0017450248712477605, P74=0.0031687382364447973, P75=0.004861627647400225, P76=0.004906905590028487, P77=0.005654557554771978, P78=0.017336881847473604, P79=0.033516635568730975, P80=0.0882735958600265, P81=0.05547135007347771, P82=0.012537084430636714, P83=0.013247738768306806, P84=0.016758642482756783, P85=0.007664517695361121, P86=0.00415111777447403, P87=0.004918072839405717, P88=0.017087364615558945, P89=0.016814404263961205, P90=0.01168629776546655, P91=0.007959331995253538, P92=0.00371488899958338, P93=0.002353642579351303, P94=0.014535484581381746, P95=0.05358742709239641, P96=0.03756939974488864, P97=0.012473636119817909, P98=0.003841870055950487, P99=0.002382546649097478, P100=0.020735848130210045, P101=0.041791023856004805, P102=0.005692065053778817, P103=0.0005394119234260697, P104=0.0013229524286927264, P105=0.012837719896909839, P106=0.014126529333730119, P107=0.002222759681466365, P108=0.0010289286222860637, P109=0.005599338240175117, P110=0.01016714395964018, P111=0.004847131959172559, P112=0.006750828709885464, P113=0.004694676245927339, P114=0.0010645941841759261, P115=0.0034017155526994526, P116=0.0009465412790821818, P117=0.0020654503770069343, P118=0.0015010466531296944, P119=0.0006626305517180187, P120=0.001041624355903353, P121=0.0003560200426297157, P122=0.000954382756446471, P123=0.0004935632618235015, P124=0.0004823645904338319, P125=0.0005680050993007502, P126=0.00035376852796781803, P127=0.0006841629980504588, P128=0.0008600424940757785, P129=0.00034754802997957594, P130=0.0003544948682230086, P131=0.0003262584823798404, P132=0.0002927237995312425, P133=0.0005129784637674775, P134=0.0009101781024667074, P135=0.0007016338569792895, P136=0.0007300238052507832, P137=0.00048604483806544043, P138=0.0002872313713461389, P139=0.00016997961906000598, P140=0.00011970545994785184, P141=7.950715053070239e-05, P142=0.00011611712699386802, P143=0.0001267932433223948, P144=0.00016085873802315572, P145=6.747858821344024e-05, P146=3.656125139502846e-05, P147=1.032375308423658e-05, P148=1.227886032732988e-05, P149=9.242931791123027e-06, P150=1.4438886762201191e-05, P151=2.840589490040593e-05, P152=7.38901428198122e-05, P153=0.0001175181066383689, P154=0.00010693944902953277, P155=0.00014574456857333187, P156=0.00019118320220300587, P157=0.0002620618311269812, P158=0.0002580280086660704, P159=0.00014595136340922764, P160=0.00013450309109633337, P161=0.00011110379073363643, P162=9.105449809431952e-05, P163=5.414425243705805e-05, P164=5.5440521873602626e-05, P165=5.0473325240622204e-05, P166=5.749625636734505e-05, P167=3.281292740090659e-05, P168=2.081859344734634e-05, P169=1.7152166439113733e-05, P170=1.0970574974463068e-05, P171=1.3747951784643359e-05, P172=2.5277275909335494e-05, P173=3.3012644516820185e-05, P174=1.721664469196741e-05, P175=1.4831779706341547e-05, P176=1.8203387353136547e-05, P177=3.260815239868937e-05, P178=2.222391704670512e-05, P179=3.057249476504848e-06),Row( P0=-469.0840232046981, P1=40.52595753486317, P2=-27.891524045425477, P3=-10.180586598799474, P4=-23.253613844130125, P5=-11.996970408662431, P6=-15.76088944119692, P7=-4.11064474394666, P8=-11.132755886824157, P9=-8.516758641613416, P10=-11.858804641563127, P11=-11.591285122748, P12=-9.356697072738783, P13=-7.500224461346089, P14=-7.337772989198652, P15=-0.23749239899066807, P16=4.777342249215868, P17=1.4945644126707762, P18=5.456351426107644, P19=1.213846265598773, P20=-3.103284201942463, P21=-1.1766801506931723, P22=0.014216445361259723, P23=-2.0726811353690957, P24=-1.2951335703349403, P25=-1.9036030242796174, P26=-0.2003845477903142, P27=-0.4638295974049297, P28=-0.21713031413996262, P29=1.4864258443469136, P30=-2.9272053301528893, P31=1.0419771298991618, P32=-0.980915982935064, P33=0.5887138184678146, P34=-0.10421730831989735, P35=2.0730841719390907, P36=-1.6219528008494655, P37=0.8900883983477769, P38=0.2536785698673468, P39=0.2970270403540477, P40=0.6923284160591183, P41=0.6088809732930782, P42=0.649209049851837, P43=0.6704284726618072, P44=0.626604289569546, P45=0.6350948125417272, P46=0.6655738345180279, P47=0.6406899400978553, P48=0.6735699960008306, P49=0.6801836988514521, P50=0.6885513124898853, P51=0.7217954270664552, P52=0.0009225535326243606, P53=0.00022383198425511526, P54=3.878991582862763e-05, P55=5.658639200030869e-05, P56=0.00031981590382552325, P57=0.0006045040058948966, P58=0.0007090898563825724, P59=0.0021351428282921025, P60=0.010784939433721627, P61=0.023189150897087646, P62=0.05148598684414461, P63=0.06714410457561427, P64=0.09211692303945836, P65=0.1600289540493924, P66=0.24976041604169835, P67=0.4373153699474099, P68=0.30775724634897467, P69=0.21539050593524092, P70=0.0950594542100594, P71=0.029549242295724157, P72=0.01190017484500918, P73=0.02475176715734948, P74=0.0228459628368202, P75=0.013252650085610475, P76=0.015537230842264943, P77=0.059361371765096344, P78=0.1761995231396936, P79=0.2988141633662443, P80=0.3137867577797418, P81=0.36264047348958767, P82=0.5701263892108225, P83=1.0301399504137687, P84=1.262789834554659, P85=0.42795195454108176, P86=0.16998998885602107, P87=0.13366517452729976, P88=0.15834429029187685, P89=0.11383568966447868, P90=0.07735376915775455, P91=0.06371320277248192, P92=0.09742417419839985, P93=0.11207203496959316, P94=0.13429763561435498, P95=0.15093896908842488, P96=0.1315283374329452, P97=0.12941229545860453, P98=0.15755903423921586, P99=0.2242982211426182, P100=0.06609547700197725, P101=0.039625727980711865, P102=0.03851944705462179, P103=0.024561937422162707, P104=0.017438270579539145, P105=0.028496759430133512, P106=0.028673683128214462, P107=0.03793493934745579, P108=0.043842619421112734, P109=0.010811762152859984, P110=0.018503063572769683, P111=0.026592760425397936, P112=0.014045550697608064, P113=0.02093258198799037, P114=0.051050765909306545, P115=0.0386441583168918, P116=0.02069909904147057, P117=0.03786406155980404, P118=0.02912876310617797, P119=0.017865020452371462, P120=0.020634606159816153, P121=0.013942740799425412, P122=0.00966090552529036, P123=0.01151001033170727, P124=0.00935134093361313, P125=0.008344622384549649, P126=0.007525092807109272, P127=0.008985317351250804, P128=0.006316595281248448, P129=0.0068855173739442496, P130=0.007413039985532667, P131=0.008887733261882226, P132=0.009467250971958307, P133=0.013106230328896287, P134=0.007301369937073519, P135=0.0052155979558380534, P136=0.005588792656863214, P137=0.002802741170854164, P138=0.00354236617320837, P139=0.001576141433829243, P140=0.001194039096328633, P141=0.001341878721045064, P142=0.0015063179780577095, P143=0.00213109450116603, P144=0.001743273655184186, P145=0.0011182372408280398, P146=0.0009816833199103399, P147=0.0008049915399035554, P148=0.0010122331271179156, P149=0.0012730025493498886, P150=0.002216081134390518, P151=0.0017339007984531441, P152=0.0014981153883878854, P153=0.0015192953487231453, P154=0.000999633094558772, P155=0.0005349593826575763, P156=0.0005153333188972814, P157=0.0006988502296346017, P158=0.0008354308366995065, P159=0.0012552841558726188, P160=0.0012719304449984434, P161=0.001097915798898092, P162=0.0012575815903002848, P163=0.0017037931067108707, P164=0.001338396916156909, P165=0.00108388108140696, P166=0.0012018244082809511, P167=0.001088581097006099, P168=0.0009791016259329277, P169=0.0010061043158880113, P170=0.000987655634189182, P171=0.0006932494389650631, P172=0.00026205043063531235, P173=0.00021666515251006298, P174=0.00020052369615020338, P175=0.00022894928274565948, P176=0.00025939444759480413, P177=0.0003973446751619197, P178=0.00023348245773832646, P179=2.2775793435291462e-05),Row( P0=-567.8637907861462, P1=55.522337009577285, P2=-23.29164907826702, P3=-0.3070081702134343, P4=-18.342660801407845, P5=-19.033504794110865, P6=-10.535498443184856, P7=3.7940841052808127, P8=-1.7477864600333006, P9=-6.561887818238291, P10=-3.7359028190606813, P11=-6.975243128904781, P12=-1.3735158710453035, P13=-8.493207369994584, P14=-4.108184558665241, P15=2.0033487487923773, P16=-0.4146380656868333, P17=1.0900605132074634, P18=-5.309294755853302, P19=-1.3621226491437508, P20=-6.195618525403993, P21=-2.0868419591456826, P22=5.266413976253778, P23=4.401485046459163, P24=7.265708555214866, P25=11.341189060171827, P26=9.4482679369817, P27=6.378171109341866, P28=4.039143974343453, P29=3.292030822030532, P30=-2.3813133154356096, P31=1.4101084665962342, P32=1.7327596984525697, P33=1.5340336802271577, P34=-1.1588012871380333, P35=-2.045112042124445, P36=-3.773742677441128, P37=-1.5728549389620858, P38=0.19909805062482694, P39=0.18074260044905824, P40=0.7180162611135231, P41=0.6809072317388097, P42=0.6169452503124587, P43=0.676590758542016, P44=0.7345318710987004, P45=0.7484202822551188, P46=0.6995619060966707, P47=0.6437969570012658, P48=0.630120001602897, P49=0.6392297452950858, P50=0.6982804557013033, P51=0.6991052319106161, P52=0.014331626523674987, P53=0.0004860641627327109, P54=1.8380932596579884e-05, P55=2.6550628475400436e-05, P56=4.555983335222766e-05, P57=3.277192815129647e-05, P58=0.0002812052441727691, P59=0.002601096720144659, P60=0.024450311033439334, P61=0.09847990252273223, P62=0.10977282009894689, P63=0.031179201514518477, P64=0.0030442125834956126, P65=0.0010226642260429935, P66=0.0016408770390796863, P67=0.003599921783165677, P68=0.004874823319888578, P69=0.014652275401371663, P70=0.04086923938749776, P71=0.07258912210757584, P72=0.11671489284013649, P73=0.11498097533422147, P74=0.03413339328840466, P75=0.005006485841034261, P76=0.0027367934300494123, P77=0.002979086320153787, P78=0.006826021735735237, P79=0.014815998856150104, P80=0.0988566200889087, P81=0.17409327215624743, P82=0.12698300908428697, P83=0.16750523024339142, P84=0.1447411070986995, P85=0.06524009150351069, P86=0.02030156667482899, P87=0.010324457495430484, P88=0.011549890305594895, P89=0.08640537749516324, P90=0.13715804436078935, P91=0.1156563412130004, P92=0.08933884886297506, P93=0.027697201402802265, P94=0.009498254221370954, P95=0.005000407386712702, P96=0.02097482175780881, P97=0.04913973267123877, P98=0.17218436067035284, P99=0.08143641933268173, P100=0.004665454702247301, P101=0.003546397245316779, P102=0.008587977919242271, P103=0.0026259855095989017, P104=0.0038725510200703756, P105=0.002777351719434587, P106=0.007138227719566197, P107=0.013475749903740897, P108=0.003924887558424047, P109=0.004851669713414012, P110=0.0074374899752557575, P111=0.033467831326417015, P112=0.026029873327385183, P113=0.03768974558307482, P114=0.011451408541725572, P115=0.005424440518196074, P116=0.011353713220250557, P117=0.016544759865584444, P118=0.0034165117333657927, P119=0.003681805758036319, P120=0.008467841536623394, P121=0.004522632530448994, P122=0.0043171733711782866, P123=0.005947104630923521, P124=0.0026150789103660827, P125=0.003097494629049593, P126=0.003995178866398591, P127=0.0019550348141906784, P128=0.0025480084698003835, P129=0.001663647239402372, P130=0.002996200667124159, P131=0.0016597143220851613, P132=0.004401092944690861, P133=0.0019517799032814562, P134=0.0024815804752596456, P135=0.0011010350697552836, P136=0.0018152284789062843, P137=0.0005436301716251004, P138=0.0012388870189749522, P139=0.0010523279358998418, P140=0.0008691362577357477, P141=0.0007083412956085642, P142=0.0005906407034369531, P143=0.0004069005904122849, P144=0.000257343154940055, P145=0.0001185746034569988, P146=0.00010424843218237254, P147=0.0001268239961858053, P148=0.00017217343640586928, P149=0.00011514856387012755, P150=9.363942250417016e-05, P151=0.0001727460274187984, P152=0.00021409256531262016, P153=0.0002616533536007325, P154=0.0002472483040003645, P155=0.000277298202370413, P156=0.00032013134759649735, P157=0.0005424823105977242, P158=0.0005526136079089177, P159=0.0006106010505108098, P160=0.0006239099130365209, P161=0.0005947711100437655, P162=0.0003269057992367675, P163=9.704460631131484e-05, P164=0.00010933790690849885, P165=0.00014482380836332298, P166=0.00020603959682503423, P167=0.00015783208201615472, P168=0.00013265699489697078, P169=0.00011791237104364984, P170=9.08538011933848e-05, P171=8.522598465639234e-05, P172=0.00013414902173927866, P173=0.00015547762818319866, P174=0.0001567973203061055, P175=0.0001378692486122698, P176=9.679993267683396e-05, P177=0.00014859674386335625, P178=0.0001391299352920841, P179=2.4464792401610963e-05),Row( P0=-571.8847865174904, P1=51.09442422475498, P2=-14.189485898328122, P3=-3.3969574364214186, P4=-9.571296648836633, P5=-14.059459169951717, P6=-8.278259884550065, P7=4.5867000596734115, P8=-10.565704342054488, P9=-5.7954496226794525, P10=-4.192047000482478, P11=-9.418173289385669, P12=2.1219961553920785, P13=-9.813883199970773, P14=-2.4812576141139284, P15=-0.3120118112879686, P16=-4.415902634172062, P17=0.7402840967206739, P18=-7.867377449282193, P19=-0.8042030449079505, P20=-3.023114007990301, P21=0.9992382982827238, P22=7.3032183600280405, P23=6.896042847259748, P24=8.762685970036015, P25=10.0241277122524, P26=3.814441887422496, P27=4.281098219272185, P28=2.74577448582717, P29=2.965500297634619, P30=2.30443800338027, P31=7.324818996337507, P32=3.279975212701857, P33=2.9637526543133257, P34=1.1498747112556325, P35=-0.8263064848083701, P36=0.21777635028100645, P37=1.531569389344458, P38=0.6806708266439043, P39=2.2356740513912516, P40=0.7062605845302526, P41=0.6298935176571081, P42=0.5920186600511992, P43=0.6211257369521362, P44=0.6967872979757246, P45=0.7662052061834287, P46=0.7119257424820361, P47=0.6574062017759813, P48=0.6228054597841168, P49=0.6124927034042825, P50=0.6121358479189117, P51=0.6378342298547042, P52=0.0042043088100515, P53=0.00012299906149329935, P54=3.626086039435051e-05, P55=7.480542436256413e-05, P56=0.00011674352889359869, P57=0.0001923546566688908, P58=0.0029355588696598728, P59=0.019411537466400067, P60=0.04191598002976779, P61=0.09241147966717832, P62=0.19740064691965292, P63=0.18137324573527966, P64=0.03502412023070454, P65=0.0031837181283260172, P66=0.0033763213146146237, P67=0.009431497928367078, P68=0.023569129378715893, P69=0.03857314862987692, P70=0.04346078498313459, P71=0.03665807443418708, P72=0.05616931888932165, P73=0.10161485406160349, P74=0.1541491653679147, P75=0.1910631418330148, P76=0.08294068169268458, P77=0.019152913813839406, P78=0.0235531112101738, P79=0.03315819172977163, P80=0.02147838184589003, P81=0.015391686816029796, P82=0.028400395378416196, P83=0.07547603656913032, P84=0.12006883679695082, P85=0.08066280169674588, P86=0.039122749788854244, P87=0.024317556340468473, P88=0.01799626212605118, P89=0.008675591483726203, P90=0.01323141897339025, P91=0.05250876218749215, P92=0.150659479697107, P93=0.06852343028488986, P94=0.009101167405931363, P95=0.004594950001231537, P96=0.0043135317899261295, P97=0.020600115673828686, P98=0.09618938017361844, P99=0.12649882203744922, P100=0.023626910756989992, P101=0.0031460544050151786, P102=0.002378954880803521, P103=0.006208627513615161, P104=0.016458522423114814, P105=0.008731910250582406, P106=0.004216873053844521, P107=0.0012264807345594652, P108=0.005867063513575076, P109=0.007160323944167681, P110=0.002579621078560289, P111=0.0025818474148070272, P112=0.007873907386385819, P113=0.015261376336890429, P114=0.002376184576736562, P115=0.0017162775460368728, P116=0.0040096988609736135, P117=0.009034043792836755, P118=0.0035214573446068396, P119=0.0022770562706590936, P120=0.0028869049093448057, P121=0.007071714085650848, P122=0.00325119599087898, P123=0.0021642475811464606, P124=0.006469155352926745, P125=0.0037246119340391694, P126=0.00439365469993685, P127=0.00543113392171346, P128=0.0017781844002603545, P129=0.0015231156918673772, P130=0.0012257822709248456, P131=0.0012397628994098389, P132=0.0009606306781252672, P133=0.0014031476858667345, P134=0.001758836756289762, P135=0.0022897326681619082, P136=0.0026308181788452217, P137=0.0012961052635601048, P138=0.0004294831632492186, P139=0.0003282344862234653, P140=0.00018506712654894577, P141=0.00021303770311313866, P142=0.00032875954295485114, P143=0.00022802207093840547, P144=0.0003550962878706366, P145=0.00021189752283879549, P146=7.697855167564188e-05, P147=2.7962617568747176e-05, P148=2.829213103521098e-05, P149=2.6766108924364e-05, P150=4.327598102658712e-05, P151=5.0127324439951646e-05, P152=0.00013245499674366038, P153=0.0001751591075217724, P154=0.00021616452650594127, P155=0.0004056535198104052, P156=0.0006563962752555282, P157=0.000857087556242398, P158=0.0015582108084620393, P159=0.0018172194714963891, P160=0.0014518523719367353, P161=0.0019917619979914524, P162=0.0015979892457513658, P163=0.00084494485360918, P164=0.0003928332946053097, P165=0.0007458590761256569, P166=0.000914851281068135, P167=0.0005862051845441667, P168=0.0004063987901119744, P169=0.0003028194418719525, P170=0.00020369763854792994, P171=0.00014347231906835527, P172=0.00016153105614616356, P173=0.00028170789001800685, P174=0.0002881632805729263, P175=0.0001567311291323579, P176=0.00012873813743475174, P177=0.00019589670775722003, P178=0.00012197663969811447, P179=1.6292765095784717e-05),Row( P0=-361.6602566641698, P1=38.61557436626205, P2=-36.271625925153444, P3=-4.612116269124259, P4=-22.73019871294769, P5=-20.488565465312785, P6=-20.51908744531877, P7=-0.04516926058238217, P8=1.0713512479041818, P9=-4.702449275574985, P10=1.5995545998627725, P11=-1.6181765868918172, P12=12.300290597137963, P13=3.5541190591753318, P14=14.431422418558832, P15=21.14557657932343, P16=12.542413303178956, P17=13.021108669763743, P18=4.478354822562695, P19=5.5838929339851076, P20=0.8840027764642081, P21=-1.4645398886309222, P22=3.631955264606651, P23=1.241284456929385, P24=-4.271616702976691, P25=3.6898007833548228, P26=-0.2336846162861252, P27=5.649151549261866, P28=6.0541134944429045, P29=3.6067672505195447, P30=-0.7538198151664312, P31=5.113862918952552, P32=-0.28482303522851965, P33=3.872317648420122, P34=0.7032814398532344, P35=3.3132111108372366, P36=0.01388415346050474, P37=0.7680217610458292, P38=-0.2183727039860729, P39=3.126882947291557, P40=0.582039638270054, P41=0.5187521184239556, P42=0.5416865541128231, P43=0.6128790980058471, P44=0.5564588890485218, P45=0.5151570107818755, P46=0.5408483685351192, P47=0.5682277750653653, P48=0.5688785770585592, P49=0.5969225402790231, P50=0.6083934422516957, P51=0.6061099271902257, P52=0.3324182176798401, P53=0.03897424374730453, P54=0.005798112265968613, P55=0.0009888075959536846, P56=0.0007103989104696165, P57=0.0005866933749528312, P58=0.00032727469204368924, P59=0.00031352547877199003, P60=0.00035433758551302525, P61=0.0005208123172062665, P62=0.002136167349459438, P63=0.006915034697435145, P64=0.010245281666852549, P65=0.06827653683963265, P66=0.4008216198528453, P67=0.7157974075076339, P68=0.9633694939319859, P69=2.057964094553422, P70=3.2130124568283134, P71=1.7345892095696875, P72=0.49683790561236957, P73=0.2750288358975998, P74=0.19383656130543478, P75=0.24480856539473017, P76=0.21422533580699085, P77=0.2533879804544433, P78=0.3500259806715255, P79=0.6142102510049445, P80=0.9404309976941245, P81=1.7914117807366958, P82=3.006009912069185, P83=2.903002914755695, P84=2.7776827781591096, P85=2.962082687913872, P86=2.954029101821888, P87=3.4100783154644887, P88=4.541414423703276, P89=2.330175483479449, P90=2.798582358230429, P91=3.640198931251646, P92=1.5593177592255993, P93=0.587761112397736, P94=1.3734182809858402, P95=1.0054004488418744, P96=0.5980870020504983, P97=4.334761668628978, P98=5.545326004508206, P99=1.1666408454578543, P100=1.746877505056789, P101=1.1111310898564515, P102=0.1015838685558134, P103=0.24728838268459824, P104=0.332983111240944, P105=1.0282362761008552, P106=0.7190383707855466, P107=0.3208911292242696, P108=1.008776818298342, P109=3.449907258000211, P110=2.5270598406264173, P111=0.44702616651018723, P112=0.35319152201124576, P113=0.8631563987840116, P114=0.610267358220978, P115=0.19069662540580853, P116=0.3304183116217192, P117=0.22512647125760765, P118=0.20625041694776466, P119=0.4144565091199862, P120=0.38886884594678045, P121=0.26683747488991805, P122=0.09805280676021393, P123=0.06717805230053105, P124=0.1612340989474648, P125=0.20636468287149462, P126=0.09717376874288833, P127=0.04603234550846761, P128=0.13289121954238237, P129=0.08682478936667572, P130=0.06789401353576316, P131=0.04325772018598616, P132=0.05545615022061634, P133=0.05821441249213325, P134=0.04965273216173973, P135=0.03740298335362543, P136=0.03537204732168923, P137=0.023789191914612644, P138=0.009658784070007663, P139=0.020636161243683712, P140=0.015892155615358925, P141=0.014844373364737172, P142=0.015626016186244263, P143=0.034719645356443236, P144=0.07158205194230376, P145=0.048958320842547476, P146=0.0446140903354822, P147=0.06368118688745161, P148=0.028336241576388744, P149=0.017361867670142295, P150=0.0326661500686766, P151=0.027951835272475873, P152=0.029851367999328454, P153=0.01739721164408836, P154=0.017356424325723162, P155=0.02092646549834526, P156=0.02068372117138518, P157=0.018674697175677754, P158=0.012299217788937238, P159=0.00864616949189659, P160=0.008368424846425356, P161=0.006628708867895046, P162=0.003847348428955741, P163=0.003035888765470344, P164=0.00325057443176955, P165=0.002403600276786649, P166=0.001994728205685875, P167=0.001658975706460293, P168=0.0014041017371940329, P169=0.0016435441017023265, P170=0.0020503781610696587, P171=0.001674645421561343, P172=0.0020456818877411316, P173=0.0022349166926522155, P174=0.0028406457646310414, P175=0.0026181925208670644, P176=0.0031970717529004504, P177=0.003750313968663649, P178=0.002012227468803157, P179=0.00022984224846153117),Row( P0=-683.3377935284477, P1=44.52759518863524, P2=-30.38284847912097, P3=-12.634353405334059, P4=-21.420795754612435, P5=-15.631096345313251, P6=-14.642704856549864, P7=-3.4229050515509063, P8=-1.1153223286647176, P9=-12.74453787585526, P10=-7.594553576253245, P11=-2.510429777839692, P12=-2.817948989540641, P13=0.9609155897791977, P14=9.704419991363613, P15=14.669238554510711, P16=8.345423045684697, P17=9.485885448986053, P18=5.110067821409697, P19=4.954997323580216, P20=2.8471173978947784, P21=2.6758162746556517, P22=-2.1753069916544185, P23=-0.1271525898075229, P24=-7.26004519144046, P25=-2.2599294385951736, P26=-2.191448954058352, P27=4.1130463729683004, P28=0.5551804037373917, P29=2.079064803848053, P30=0.5990230383066193, P31=4.0172594324531, P32=-1.028212402200525, P33=-0.7225133195310154, P34=-3.250294227071631, P35=-1.1487109492466663, P36=-0.6740803414119743, P37=3.212619277327455, P38=-0.007107332316100286, P39=-0.12181850977340777, P40=0.660048728772327, P41=0.656812143180658, P42=0.6270350926641194, P43=0.5145611479522486, P44=0.4813554209409334, P45=0.43586787239381963, P46=0.4384416769049419, P47=0.49147146220260296, P48=0.595401170243696, P49=0.6411079162342722, P50=0.6112927282512681, P51=0.6287268810280893, P52=4.970105792770229e-06, P53=2.4528154060918677e-06, P54=8.080534916871339e-07, P55=1.2874809103471018e-06, P56=1.0679246596963072e-06, P57=8.624471785983382e-07, P58=5.36975513415861e-07, P59=9.039189989313076e-07, P60=1.0342428226970972e-06, P61=2.2070132069917693e-06, P62=1.1869550173239777e-05, P63=0.00023858152183211316, P64=0.003296802962154325, P65=0.01298306743626416, P66=0.014778352860070423, P67=0.015456639660358956, P68=0.026413331317253614, P69=0.059682091147321134, P70=0.04089667491920846, P71=0.00951779038554627, P72=0.0011582981751388445, P73=0.00039712278462274885, P74=0.00016568635212545245, P75=0.00017155872050497156, P76=0.000264291244782497, P77=0.004342738453896115, P78=0.018048027895676815, P79=0.016432449224606877, P80=0.005820662922894697, P81=0.0021717237720069193, P82=0.0012181572260962737, P83=0.0022308645826086197, P84=0.012359865460220144, P85=0.024761881176920308, P86=0.016532680554334173, P87=0.01983535192968712, P88=0.01903821774992668, P89=0.010712190694682626, P90=0.005846666699273257, P91=0.005914327724246615, P92=0.002223126335325279, P93=0.0007126058261804059, P94=0.0008241559953398781, P95=0.0009529252102651899, P96=0.00036073814465382016, P97=0.0002373839154433838, P98=0.0007983890979301265, P99=0.0010157855693519772, P100=0.0012892861614011277, P101=0.0014575524175877415, P102=0.0017204594417386634, P103=0.0005889269620116704, P104=0.0001958297883543956, P105=7.913766232198956e-05, P106=0.00017431376940816055, P107=0.000781541198974455, P108=0.0013326431620498064, P109=0.001359277568852624, P110=0.0009795611764308327, P111=0.0003770600053437809, P112=0.0005137078587604719, P113=0.000200185129927809, P114=0.00026240688515026886, P115=0.0004039979773766956, P116=0.00047428612342172117, P117=0.0007257559487195127, P118=0.000638346421437639, P119=0.0007321969951495402, P120=0.0003315136577543921, P121=0.00040679418344847503, P122=0.00016358475143164508, P123=0.00017939744331010177, P124=0.00014869592449403868, P125=0.0001239392842931194, P126=0.00012269527419793896, P127=0.00017084020675060534, P128=0.00010375510205660775, P129=4.211269597175418e-05, P130=3.984038300524488e-05, P131=7.162772859989448e-05, P132=1.670099526761076e-05, P133=1.7791112094629304e-05, P134=3.211849536574697e-05, P135=3.64891510206378e-05, P136=1.719749399812602e-05, P137=4.669470710596446e-05, P138=2.8671845471709208e-05, P139=2.226799103409343e-05, P140=4.8495546877107614e-05, P141=7.305517091030679e-05, P142=7.636973682246647e-05, P143=5.8205619150474355e-05, P144=2.0878537175877597e-05, P145=1.2429116967081517e-05, P146=1.4062046363213372e-05, P147=1.4545388813970455e-05, P148=1.8659465224916688e-05, P149=1.4411616728017147e-05, P150=1.7351209837235414e-05, P151=2.6354054382793687e-05, P152=3.968131724658484e-05, P153=3.097274114936233e-05, P154=2.3761336039599963e-05, P155=1.7907484383753034e-05, P156=2.9044333132017887e-05, P157=3.147488867287451e-05, P158=3.950901122444959e-05, P159=7.255458628081153e-05, P160=0.00011911918290689702, P161=0.00012789334116847223, P162=3.262448118599328e-05, P163=5.5994961813366845e-05, P164=4.873852068486462e-05, P165=3.724501482216581e-05, P166=0.00010771001482311578, P167=0.0001457828093856598, P168=0.00010228486437127524, P169=0.00015012998364684693, P170=9.784141478949084e-05, P171=4.7345878546877807e-05, P172=5.135200259609644e-05, P173=9.114957023331645e-05, P174=0.00011257787344650134, P175=8.038299376636543e-05, P176=4.259995585085268e-05, P177=3.8419746550331394e-05, P178=1.3576593638713865e-05, P179=1.1256975564221817e-06),Row( P0=-578.5290204588247, P1=18.159119371814388, P2=-23.24542616844636, P3=-10.230360328669132, P4=-18.54272004714492, P5=-15.320137722312815, P6=-13.478124676537671, P7=-10.364597436704162, P8=-0.7471453575010223, P9=-3.566891601610552, P10=1.74432505657919, P11=12.852935128184889, P12=10.554248821593276, P13=7.909205095242986, P14=3.289456039963533, P15=-1.9978900364730934, P16=-11.094932276577305, P17=-3.8237932512528188, P18=-3.205501319939157, P19=-0.734589487390567, P20=-1.0468381488335292, P21=-2.899458350611317, P22=-2.1372970720197557, P23=2.8512116195879895, P24=0.7613396614805825, P25=-0.9450495603336051, P26=-5.452834507029701, P27=-0.7365553964646804, P28=-0.5701631390428858, P29=-0.128810573802396, P30=-1.9951116718407063, P31=1.2163723222606415, P32=-1.0263443512780548, P33=-0.24742859207832388, P34=-2.4105448035453927, P35=0.344269956020176, P36=0.08398768147801268, P37=-1.689661787512288, P38=-2.8495796792239965, P39=-0.4182754460581657, P40=0.4262530578107581, P41=0.42580516808116414, P42=0.45725113763991115, P43=0.45871859401325615, P44=0.4867752046219298, P45=0.505814832879532, P46=0.5201692129202872, P47=0.5693841054222881, P48=0.46282111753389443, P49=0.4263619834436288, P50=0.4380294362754622, P51=0.4293427294984318, P52=1.4351316321671946e-06, P53=2.1264056870906165e-06, P54=1.0648435754888743e-05, P55=2.3991184570919747e-05, P56=2.484384789859132e-05, P57=2.15517565621476e-05, P58=1.9317866744680996e-05, P59=2.034032183457196e-05, P60=1.7301503157285824e-05, P61=1.9078350645535268e-05, P62=1.9105677883960212e-05, P63=1.4881392051193807e-05, P64=1.685280128145148e-05, P65=0.0003107500908924355, P66=0.003800391986409472, P67=0.008791973918212882, P68=0.01299704525497036, P69=0.07693999561654911, P70=0.17819592867629008, P71=0.16621449318943454, P72=0.260631442829995, P73=0.25905150934469645, P74=0.7310409117766355, P75=1.2332405240673747, P76=0.4887405050249816, P77=0.13369732110318977, P78=0.02811955852270892, P79=0.008530100193147618, P80=0.0023851550651637276, P81=0.000796103641011221, P82=0.0004232938156190816, P83=0.00047707862963975345, P84=0.0007114145530131492, P85=0.0008981327267581656, P86=0.004516755290575276, P87=0.03619378671544428, P88=0.09444577776547415, P89=0.06135205298236386, P90=0.03609535095255527, P91=0.06237589529038427, P92=0.09816319332460342, P93=0.08434042795337839, P94=0.15522004738182812, P95=0.2125242975606563, P96=0.10647237398566232, P97=0.02914246848323557, P98=0.004224901535755998, P99=0.00226367906179634, P100=0.009405962287400213, P101=0.02173092829595378, P102=0.03372782494175702, P103=0.030114918948613914, P104=0.050489131637035056, P105=0.025558459935999048, P106=0.023797982899755533, P107=0.04296211497264301, P108=0.03826235342268391, P109=0.01331759412558099, P110=0.004994185886273552, P111=0.013943586674665992, P112=0.011991859339314676, P113=0.005549110249069416, P114=0.007916990486950982, P115=0.02175733303492711, P116=0.0327286078851748, P117=0.06975629634670284, P118=0.04092413521534278, P119=0.0067812550508273214, P120=0.001854502240405187, P121=0.002664083357736338, P122=0.004160189707522128, P123=0.02602993137579019, P124=0.03297427024986402, P125=0.013983789897791346, P126=0.0014140618328355443, P127=0.0008522920607393221, P128=0.0033781420003710606, P129=0.0023889001503842088, P130=0.0007114153626373134, P131=0.0002592817485123155, P132=0.0005011564560135524, P133=0.00048323108999233746, P134=0.0013960003040713945, P135=0.0006629553800040015, P136=0.0005199942768521443, P137=0.00041700502321557033, P138=0.0011684712046420266, P139=0.0015095048000362916, P140=0.0005220707815939154, P141=0.0009869397134564634, P142=0.0018856269412585694, P143=0.0008995028613166804, P144=0.00061067463295588, P145=0.0006409861945310429, P146=0.00038377382663267357, P147=0.0002098111290369321, P148=0.0002648715472231399, P149=0.00036295595746983783, P150=0.0002663444781142977, P151=0.0003814928056539645, P152=0.00038953555800539286, P153=0.0005763651313801706, P154=0.0004904736725707093, P155=0.0004088490877301412, P156=0.0002641090814446896, P157=0.0003493140174968222, P158=0.00042245100910877657, P159=0.0005798588887146156, P160=0.0008265442972958808, P161=0.0005618711626315397, P162=0.00023383946882691109, P163=0.0002268708955019284, P164=0.0003345099398420164, P165=0.0004386969201059798, P166=0.0006400511905983693, P167=0.000381522548544375, P168=0.0005759834488390178, P169=0.0007431981484808579, P170=0.0005982048475490239, P171=0.0005335569986900146, P172=0.0006722403878617172, P173=0.000952829090647615, P174=0.001866576468826222, P175=0.0015675817937633493, P176=0.0014368353905485506, P177=0.0010006086548283314, P178=0.0005582135857112469, P179=9.063663531880413e-05),Row( P0=-496.77082354346584, P1=11.254087306622994, P2=-36.78680903158408, P3=-8.221283416372643, P4=-25.44621652422734, P5=-18.18340450091178, P6=-10.841609410019618, P7=-6.331645130796629, P8=2.114751158166689, P9=-1.0914544561740418, P10=11.199501395184305, P11=10.982512806267511, P12=1.4777604010248229, P13=-2.837968176847801, P14=-0.8494745001139978, P15=-2.252576751967032, P16=-7.705221673250563, P17=2.928040825458962, P18=-2.575051713773412, P19=0.8983791617941417, P20=-0.8046834658869653, P21=0.2851009165176353, P22=-0.4966402735793143, P23=-0.8502349755920954, P24=-2.2698166281957803, P25=-1.169391759951496, P26=-2.7760633055982216, P27=0.11366102420820484, P28=0.07856640977450387, P29=1.5557508680946668, P30=0.5032018857117627, P31=1.64241998493549, P32=-0.41593966377415015, P33=0.5661866235327001, P34=-1.0450561690407716, P35=-2.0623444027307087, P36=-1.732308222301778, P37=-0.4203614913987083, P38=-1.1939868821435267, P39=-0.2584574646161682, P40=0.661834203438424, P41=0.6405629629233713, P42=0.6074203545452724, P43=0.5662264737040514, P44=0.578542900410111, P45=0.6037977242966026, P46=0.6159353638185021, P47=0.6447735454269942, P48=0.6617728062129836, P49=0.7326660434336655, P50=0.7171665118481869, P51=0.6537804932591063, P52=0.00022943779472056004, P53=0.00017757523010721523, P54=6.41757480867218e-05, P55=4.2032507320759566e-05, P56=8.525144840049352e-05, P57=0.00011981280982354401, P58=8.45348416325956e-05, P59=9.133170808123058e-05, P60=0.0001286702168658203, P61=0.00018689667916529966, P62=0.00029466147586625644, P63=0.0002478287469871934, P64=0.00012874414650542175, P65=0.0003948013779584664, P66=0.0017278892215846263, P67=0.005911706800446706, P68=0.04981789897019344, P69=0.1351423074252174, P70=0.08983742561533638, P71=0.0603105851479374, P72=0.09381699525935218, P73=0.19022645763735618, P74=0.5492426228514874, P75=0.883115775034857, P76=0.862502082674357, P77=1.4534800455392505, P78=1.22967675097789, P79=0.8757082454504991, P80=0.5557227487417157, P81=0.3581268270103623, P82=0.2381858970873761, P83=0.3010302191566748, P84=0.607280478194093, P85=0.6208754763161257, P86=0.7524228824583988, P87=0.9942397128440433, P88=0.5515404819534279, P89=0.16628472693460308, P90=0.07024749915272317, P91=0.1099398913297205, P92=0.14082424880210603, P93=0.21007443456733021, P94=0.33430324688645663, P95=0.5853212314044457, P96=0.5112108986032083, P97=0.7301142096601375, P98=0.9945431478019444, P99=0.4629054442152339, P100=0.4248295805319327, P101=0.21782336141848618, P102=0.21909376058181854, P103=0.22334987499383452, P104=0.12134673590616872, P105=0.5461196505421747, P106=0.5146485000278384, P107=0.11415694877682109, P108=0.1127477941759588, P109=0.10517044259530699, P110=0.2825403225525263, P111=0.28779648293073495, P112=0.2943994064155426, P113=0.13679865785791226, P114=0.09994689039797124, P115=0.10934926708656846, P116=0.15361382206959018, P117=0.499989238329337, P118=0.3182922492793673, P119=0.15170997506886177, P120=0.19862759866415464, P121=0.4038231360297976, P122=0.28595868853351575, P123=0.14966159341988747, P124=0.24584026659118166, P125=0.19894557654890915, P126=0.05748659935853891, P127=0.035149423457911076, P128=0.07249041606414906, P129=0.04044361052208997, P130=0.0379853037855299, P131=0.021483222501646367, P132=0.027328601855884142, P133=0.026846946555076856, P134=0.0182372220032362, P135=0.014736066960120022, P136=0.026464624627238167, P137=0.02050644836772262, P138=0.019350559201806228, P139=0.01962567876458399, P140=0.02790710650307904, P141=0.036494934788604695, P142=0.03214562468159629, P143=0.04416725434006285])
'''

In [ ]:
#print(type(y))
#newy=np.array(y)
#newy = newy.reshape(768,1)

#z=x.copy()
#z = np.append(z, newy, axis=1)
#print(z)

##numpy to pd
'''
x1 = pd.DataFrame(x)
print(x1)
y1 = pd.DataFrame(y)
print(y1)
'''
#z=pd.DataFrame(z)

##pd to pyspark.DataFrame

#spark.conf.set("spark.sql.execution.arrow.enabled", "true")
'''
x2 = spark.createDataFrame(x1)
print(x2)
y2 = spark.createDataFrame(y1)
print(y2)
'''
#z= spark.createDataFrame(z)
'''
#print(x_train)
print(y_train)
#finaldf=[]
print(len(x))
print(len(y))
for i in range(len(x)):
  finaldf.append((x[i],y[i]))
print(finaldf)
dfForVA = spark.createDataFrame(finaldf, ["x", "y"])
assembler = VectorAssembler(inputCols=["x","y"],outputCol="features")
ll=assembler.transform(dfForVA)
print(ll)
'''
#print("z is",z)
#import sklearn.pipeline as pp
#assembler = VectorAssembler(inputCols=["P0", "P1", "P2", "P3", "P4", "P5", "P6", "P7", "P8", "P9", "P10", "P11", "P12", "P13", "P14", "P15", "P16", "P17", "P18", "P19", "P20", "P21", "P22", "P23", "P24", "P25", "P26", "P27", "P28", "P29", "P30", "P31", "P32", "P33", "P34", "P35", "P36", "P37", "P38", "P39", "P40", "P41", "P42", "P43", "P44", "P45", "P46", "P47", "P48", "P49", "P50", "P51", "P52", "P53", "P54", "P55", "P56", "P57", "P58", "P59", "P60", "P61", "P62", "P63", "P64", "P65", "P66", "P67", "P68", "P69", "P70", "P71", "P72", "P73", "P74", "P75", "P76", "P77", "P78", "P79", "P80", "P81", "P82", "P83", "P84", "P85", "P86", "P87", "P88", "P89", "P90", "P91", "P92", "P93", "P94", "P95", "P96", "P97", "P98", "P99", "P100", "P101", "P102", "P103", "P104", "P105", "P106", "P107", "P108", "P109", "P110", "P111", "P112", "P113", "P114", "P115", "P116", "P117", "P118", "P119", "P120", "P121", "P122", "P123", "P124", "P125", "P126", "P127", "P128", "P129", "P130", "P131", "P132", "P133", "P134", "P135", "P136", "P137", "P138", "P139", "P140", "P141", "P142", "P143", "P144", "P145", "P146", "P147", "P148", "P149", "P150", "P151", "P152", "P153", "P154", "P155", "P156", "P157", "P158", "P159", "P160", "P161", "P162", "P163", "P164", "P165", "P166", "P167", "P168", "P169", "P170", "P171", "P172", "P173", "P174", "P175", "P176", "P177", "P178", "P179", "P180"],outputCol="features")

#pipeline1 = pp.Pipeline([("assembler",assembler),("mlpc",mlpc)])

#answer = pipeline1.fit(z)
#print(answer)




#models = mlpc.fit(z)